In [35]:
import dhlab.nbtext as nb
import dhlab.module_update as mu
import json
import requests
#import Levenshtein as lev
mu.css()

In [37]:
!pip install fuzzywuzzy

In [115]:
!pip install python-Levenshtein

  Stored in directory: C:\Users\Eier\AppData\Local\pip\Cache\wheels\de\c2\93\660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [116]:
from fuzzywuzzy import fuzz

In [22]:
def get_paragraphs(urn, paras):
    """Return paragraphs for urn"""
    param = dict()
    param['paragraphs'] = paras
    param['urn'] = urn
    r = requests.get("https://api.nb.no/ngram/paragraphs", json=param)
    return r.json()

In [23]:
def get_para(para_list):
    import ast
    paras = []
    for p in para_list:
        paras.append(ast.literal_eval(p))
    urns = [x[0] for x in paras]
    paradict = {u:[x[1] for x in paras if x[0] == u] for u in urns}
    paragraphs = dict()
    for u in paradict:
        paragraphs[u] = get_paragraphs(u, paradict[u])
    return paragraphs


In [26]:
themes = json.load(open('heidis_temadefinisjoner.json'))

In [51]:
link = lambda x: "https://urn.nb.no/URN:NBN:no-nb_digibok_" + str(x)

In [31]:
for th in themes:
    print(th)

“Jul” [Christmas]
“Kvindens funksjon i den private sfære” [Woman’s function in the private sphere]
“Kvindens funksjon i den offentlige sfære” [Woman’s function in the public sphere]
“Kvinnelig dannelse” [Woman’s bildung]
“Kvinnens væren” [Woman’s being]
“Er kjærligheten sann” [Is love true?]
“Kvinnens beskyttelse” [Woman’s protection]
“Kvinnens mot under prøvelser” [Woman’s courage during difficult times]
“Båndet mellom kjønnene” [The bond between the genders]
“Slavinnen og herren” [(Female) Slave and master]
“Tjenerinnes romantiske forbindelse” [the Female servant’s romantic liaison]
“Kvinnes selvmord” [Woman’s suicide]
“Kvindens vilje” [Woman’s Volition]
“Innflytelse over kvinnen” [Influence on Woman]
“Et fruentimmers dyd” [Woman’s virtues]
“En kvinnes væren og tillærte egenskaper” [Woman’s being and acquired traits]
“Kvindens stillhetens språk” [Woman’s language of silence]
“Mill, en frigjører eller rabulist?” [Mill, a liberator or an offender]
“Kvindens likegyldighed til frihetsret

In [53]:
print(themes[find_theme('Jul', themes)])

{'Mor_': ['Mor', 'Mødre', 'moderlig', 'moderligt', 'Modcr', 'modcr', '', 'Modre', 'Msdre', 'Msdrc', 'modre', 'modcrlig', 'modrcne', 'modrc', 'modcrlig', 'modcrlighcden', 'modcrlighed', 'moderlighed', 'modcrlighcd', 'Huusmoder', 'husmoderlig', 'husmoderlige', 'husmoderlighed', 'husmoderligheden', 'Husmor', 'Husmoder', 'Husmodcr', 'Husmodcrcn', 'Husmoderen', 'Husmodcren', 'Husmodcren', 'Huusmodcr', 'Huusmodre', 'Huusmoders', 'Huusmodcr', 'Huusmodrene', 'Huusmodcren', 'Huusmodercn', 'Huusmodcrs', 'Husmodre', 'Husmodrc', 'Husmoderem', 'Husmoderms', 'Huusmodcrcn', 'Moderlig', 'Moderligt', 'Modcrlig', 'Modrcne', 'Modrc', 'Modcrlighcden', 'Modcrlighed', 'Moderlighed', 'Modcrlighcd', 'Husmoderlig', 'Husmoderlige', 'Husmoderlighed', 'Husmoderligheden'], 'Kvinde_': ['kvinne', 'kvinnen', 'kvinner', 'kvinnene', 'kvinnerne', 'kvinnes', 'kvinnens', 'kvinners', 'kvinnenes', 'kvinnernes', 'kvinde', 'Kvmde', 'kvinden', 'kvmden', 'kvinder', 'kvinderne', 'kvindens', 'kvmdens', 'kvindes', 'kvmdes', 'kvind

In [117]:
import os
r,d,f = next(os.walk("."))
jsons = [fs for fs in f if fs.endswith('.json')]
files = {f:find_theme(f, themes) for f in jsons if find_theme(f, themes) != ''}

In [118]:
files

{'Er kjærligheten sann Is love true.json': '“Er kjærligheten sann” [Is love true?]',
 'Jul Christmas.json': '“Jul” [Christmas]',
 'Kvindens funksjon i den offentlige sfære Womans function in the public sphere.json': '“Kvindens funksjon i den offentlige sfære” [Woman’s function in the public sphere]',
 'Kvindens funksjon i den private sfære Womans function in the private sphere.json': '“Kvindens funksjon i den private sfære” [Woman’s function in the private sphere]',
 'Kvinnelig dannelse Womans bildung.json': '“Kvinnelig dannelse” [Woman’s bildung]',
 'Kvinnens væren Womans being.json': '“Kvinnens væren” [Woman’s being]'}

In [101]:
def find_theme(title, themes):
    from collections import Counter
    dist = Counter()
    for th in themes:
        dist[th] = fuzz.ratio(th, title)
    x = dist.most_common(1)[0][0]
    if dist[x] > 60:
        res = x
    else:
        res = ""
    return res

In [112]:
def show_paras(title, themes):
    avsnitt = json.load(open(title))
    paras = get_para(avsnitt)
    theme = find_theme(title, themes)
    words = [w for hyperlexeme in themes[theme] for w in themes[theme][hyperlexeme]]
    ptext = "<h2>" + theme + "</h2>"
    ptext += "<h3" + ', '.join(themes[theme].keys()) + "</h3>"
    for urn in paras:
        ptext += "<h4> <a href='" + link(urn) +"'" + "target=_>" + ', '.join([str(x) for x in nb.metadata(urn)[0][1:4]]) + "</a>"
        for paranum in paras[urn]:
            ptext += "<h4>" + paranum + '</h4>'
            modified = paras[urn][paranum]
            #print(modified)
            for w in words:
                #print(w)
                modified = modified.replace(" "+w+" ", " <b>" + w + "</b> ")
            ptext += modified + '\n\n'
    return ptext

In [119]:
res = ""
for f in files:
    print(f)
    res += show_paras(f, themes) + "\n"
#mu.printmd(res)

<h2>“Er kjærligheten sann” [Is love true?]</h2><h3Kvinde_, Kjærlighed_, Sann_, Selv_, Hjerte_, Øye_</h3><h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010061503024'target=_>Scriver, Christian, 1858, ster Christian Scrivers opbyggelige og trøstefulde Sjele-Skat</a><h4>394</h4>§ 31. Om den beromte tydste Keiser Karl den Femte fortaller Historien , at han efter Katholikerues Viis pleiede at pidste sig <b>selv</b> , og at hans Son Filip den Anden , Konge af Spanien , opbevarede den Pidst som et dyrebart Klenodie , hvormed hans Fader haode hndstettet sig og som var farvet og bcstanket med hans Blod , samt paa sit Iderste overleverede den til sin Soll , Filip den Tredie . Opbevar ogsaa dn , o Ehristensjel , formedelst en bestandig Ihukommelse de Riis og Pidste i dit <b>Hjerte</b> , der ere farvede med Jesn dyrebare Blod , og hold dem srem for dit syndige Kjod og betving det dermed , saa ofte det faaer Lyst til Synden . § 32. Betragt Ham dernast , hvorledes Han hanger paa sit Kors og bloder sig tildode . Jeg holder sor , at et eneste <b>Blik</b> paa den korsfasiede Frelser , nåar det steer i Tro og <b>sand</b> Andagt, er tilstratteligt til at dampe al Syndelyst . I Florents i Ilalien tildrog det sig , at en betjendt gudfrygtig Munk boede i Narhcden af en ntyst <b>Kvinde</b> og onstede at omvende hende fra hendes onde Veie og fore hende til den Herre Jesum . Til den Ende udtanter han folgende Middel . Han lader deu torsfastede Jesus afmale i en ret begradelig Skikkelse paa en Tavle , som han derefter lader indfatte som et Speil og ophanger i et Vindn ,

<h4>960</h4>Rost bagved sig : Hvorfor vil du doe , du forforte og forblindede <b>Hjerte</b> ? Jeg har ikke Behag i hans Dod , som doer . Omvend dig dog ; saa skal dn leve . Hvad har jeg gjort dig , mit Barn , og hvormed har jeg gjort dig Moie ? Svar mig ! ( Mik . 6 , 3 ) ' Kjare Barn , hvor vit dn da lobe hen ? Til hvilket Sted vil dn flye hen , hvor ikte jeg stnlde vare ? Farer dn op til Himmelen , saa er jeg der , og reder dn Seng i Helvede ( i det Dybe ) , saa er jeg ogsaa der ; tager dn Morgenrodens Vinger og vil boe ved det yderste Hav , saa vilde dog ogsaa der min Haand finde dig . Derfor , kom til mig ; erkjend min <b>Kjarlighed</b> og Naade , hvormed jeg <b>moder</b> dig i alle Skabninger . Har du syndet , saa er hos mig megen Naade ; har du forladt mig , saa har dog min <b>Kjarlighed</b> og Trostab ikke forladt og forstodt dig , men har gaacr efter dig , har sogt dig og kalder paa dig . 8 21. Det er en elskelig Tildragelse , som nogle gamle Larere fortalle og som vi ikke knnne afholde os fra at meddele , efterdi dcli er meget tjenlig for vort Forehavende . Apostelen Johannes havde anbefaler et vattert nngt Menneske til en asiatisk Biskop , foråt han skulde undervise ham i Chriftendommen og veilede ham til <b>sand</b> Gndsfrygt . Dette , gjorde nu Vistoppen og bragte det dertil , at den nnge Mand ved den hellige Dnab blev indlemmct i den christelige Kirke . Senere havde han sormodcntlig tilstedet ham storre Frihed end tjent igt var , saa at Unglingen , kort at fortalle , kom i ondt Selstab og derved blev forfort til ' et vildt og ndsvavende Levnet , ja tilsidst flog sig til Stratenroverne og blev deres Hovedsmand og Anforer . Da nn Apostelen efter nogle Aars Forlob kom tilbage der til Staden , mindedes han den unge Mand og spurgte efter ham , men fik det sorgelige Svar , at han desvarre havde stikket sig ilde , og at det stod til med > ham som nysmeldt . Herover blev Apostelen hjerteligen forfardet og bedrover , men begav sig af christelig , brandende <b>Kjarlighed</b> nden lang Betamming afsted sorat opsoge det tabte Faar . Han tom til dell Egn , hvor man havde sagt ham at Roverne opholdt sig , lod sig godvillig tåge tilfange af disse og bad om at blive bragt til deres Hovedsmand . Da denne fik <b>Oie</b> paa ham og gjenkjendte ham , begyndte han

<h4>2126</h4>indveute : jeg tvivler vel Me om Guds Naade og Barmhjertighet » ; jeg veed jo ogfta , at Guds Sous Blod er en overflodig Betaling for Alverdens Synder; men jeg tvivler om mig <b>selv</b> , om jeg er skikket til at modtage Guds Naade og Syudernes Forladelse ; thi min Omvendelse , synes mig , er ikke <b>sand</b> og ikke saadan , som Gnd i sit Ord fordrer den . Jeg soarer : det er allerede et godt Tegn til en <b>sand</b> Omvendelse , nåar man er omhyggelig for sin Omvendelse . Hyklerne og de Ugudelige have let for at blive ferdige med sin Omvendelse vg give sig derfor snart tilfreds ; de sande Bodferdige derimod knnne ikke gjore sig selo tillags , og det gjor dem ondt , at de Me knnne angre sine Synder fta hjerteligt , som de onste og ville . Den sande Omvendelse er imidlertid let at kjende . Hvvr der er en inderlig Bedrovelse over Synden , en lengselsfuld Attråa ester Guds Naade , en bestandig Sukken til Ehristus , et oprigtigt Had og Fiendstab mod Synden , der bor man ikke tvivle om Omvendelsens Oprigtighed . Og om end disse Omvendelsens Bestanddele endnu Me ere komne til saadan Fuldkommenhed , som de vistnok burde , fta vide vi dog , at selo det bodferdige <b>Hjertes</b> forste Lengsel og forste Bevegelser ere vvr Gud velbehagelige, ligesom eu Huusftder allerede gleder sig i et Trees Frugtknupper , soreud de spriuge ud . Da den fortabte Son endnu var langt borte , ftae hans Fader ham og ynkedes inderligen og lob til og ftldt om hans Hals og kyssede ham ( Luk . 15. 20 ) , hoorved intet Andet betegnes end dette : medens vi endnu agte os uverdige til uogen Naade og <b>Kjerlighed</b> og tenke , at vi endnn ere langt borte fm Ham og neppe have begyndt at vandre paa Omvendelsens Vei , fta <b>moder</b> allerede den barmhjertige Gud os og tager os til Naade . § 35. Et ftadcmt <b>Hjerte</b> siger endvidere : Jeg er haard som Steen og Staal ; jeg frygter ikke rigtigt for Gnds Vrede ; jeg skjelver ikke for Hans Ord ; jeg er ikke sonderreven , sonderslagen , sonderknnst , hvilket jeg dog ifolge Skriften stal vere ; jeg kanikke bederigtig andegtigt , ingen hjertelige Taarer ndgyde; jeg er endnu letsindig og kold som for , osv. Jeg svarer : Giver for det Forste Agt paa Forskjellen paa Eders nuverende og Eders forrige Tilstand og sammenligner dem med hinanden ! Forhen bekymrede I

<h4>2464</h4>i dunkle tuen liflige Ord ( Hois . 2 , 3 — 6 ) . Hun sammenligner stn Elster med et frugtbart LEbletre , der ei alene yter hende Skygge mod Heden , men ogfta vederkveger hende med sine sode Frugter , hvilket ikke kau forklares om noget Andet end om den Herres Jesu veldige Bestjermelse vg kraftige Trost . Hun siger fremdeles : Han forer mig til Viinhuset, det er : Han gjor mig fuld af Aanden ( Eses . 5 , 18 ) og lader mig fole sit Ords Kmft i mit <b>Hjerte</b> , - Han gjor mig drtMeu af lutter <b>Kjerlighed</b> og Naade , - Han gyder gjennem sit Ord ftamegen Trost i mit <b>Hjerte</b> , at jeg glemmer al min Sorg og lader Hans Naade vere mig nok , — og <b>Kjerlighed</b> er hans Banner ( Hans flyvende Fane ) over mig ; den Forsikring om og Fornemmelse as Hans <b>Kjerlighed</b> , som jeg modtager af Hans hellige Ord og hoiverdige Sakrament , er for mig og mine Medchristne ligesom en Fane , hvornnder vi stride manteligen imod Synd , Verden , Dod , Djevel og Helvede og vinde den ene Seier ester den anden ( Ps . 84,8 ) . Hatt vederkveger mig med Balsomflaster , reder nnder mig med LEbler ; thi jeg er syg af <b>Kjerlighed</b> ; Meningen er : Efterdi jeg formedelst den Forsmag , jeg har ftaet paa Hans Godhed , stedse foler storre og storre Lengsel ester Ham og higer fta inderligt efter Ham , at det nesten gjor mig syg , fta lader Han sine Tjenere vederkvege vg vpreise mig med de edleste og dyrebareste Sprog af Skriften , der overgaae Alvertens Blomster og Mbler i Vettugt og Velsmctg . Haus venstre Haand ligger under mit Hvved , og Hans hoire Haand omsa tiner mig ; Hun taler , som ett Brud stal tale , og siger : jeg hviler i min Herres Jesu Kjerligheds Arme ; deri vil jeg leve vg doe ; Haus venstre Haand er liig en blod Pute under mit trette og sorgfulde Hoved ; med sin hoire Haand torter Han Tactrerne af mine <b>Oine</b> , og ved sin kraftige Bistand og nctadige Hjelp i al Nod soroisser Han mig om sin <b>Kjerlighed</b> . Slige Ord og overhodet » den heleHoiftng kan intet kjodeligt , jordistsindet <b>Hjerte</b> sorstctae , og jeg kan med al Ret sige derom , hvad den Herre lesns <b>selv</b> siger om det nye Navn , sont Han giver sine Troende , at Ingen kjender det nden den , som annammer det ( Actb . 2 , 17 ) . Slige Ord , som den hitnmelste Brud her taler , kan Ingen forstaae uden den , som lever i en <b>sand</b> Forening

<h4>2599</h4>modig , den Hellig Aands Drift og Tiltale , der opholdt ham i Troen og <b>Kjerligheden</b> til den Herre Jesus indtil Doden . § 26. Disse Exempler ere ftare udnterkede , og mangen eenfoldig Christen vil maastee tenke , at de ere aldeles enkeltstactettte . Vi ville derfor ogfta anfore nogle andre . Erfaringen lerer , at sorgmodige og anfegtede Mennester ofte hos sig <b>selv</b> fornemme ligefom en Samtale mellem to stridende Parter , hvoraf den ene klynker og klager , den anden troster og styrker . I den gytdne Pftlmernes Bog er dette paa forstjellige Steder meget smnkt fremstillet . Hvorfor er dn nedboiet , min Sjel , og brnser i mig ? Bi ester Gud ; thi jeg skal endnu prise Ham , mit Ansigts Frelse og min Gud ( Ps . 42 , 6. 11 ) . David har i det Foregaaende omtalt sin Sjels Bedrovelse vg skildret dens Urolighet); nu troster han den ved en Tiltale , som den Hellig Aand giver ham i Sinde . Begyndelsen tit den trevgsvttiende Psalme viser , at Profeten , for han forfattede samme , havde siddet i dybe og sorgmodige Tanker . Hans Kjod og Blod sagde : Er jeg et Guds Bant , hvorfor gaaer det da mig fta ilde og de Ugudelige fta godt ? Den Hellig Aand svarer : Ak , jeg vil dog haabe , at jeg er et Guds Bant , siden jeg har de herlige Forjettelser i Hans Ord og er tmadt i Pagt med Ham ved Omstjerelsen , osv. Kjodet vedbliver : Imidlertid have vi dog hver Dag Korset paa vor Nakke og maae daglig ste de Ugudeliges Overdacidighed for vore <b>Oine</b> . Aanden svarer , styrket af den Hellig Aand , som forener sig med den : Har da ikke Gud stedse og altid behandlet sine kjereste Born saaledes ? Ligger der Me stjnlt en Guds Kjerligheds Hemmelighed uuder Korset ? Skal matt da blot domme efter det udvortes Udseende og Begyndelsen og Me efter den indvortes Bestaffenhed og Udfcildet ? Ere da Verdens forgjengelige Ting et Kjendemerke paa Guds Born , eller kunne de skjenke nogen <b>sand</b> og varig Lyksalighed? Herpaa folger den Slutning : Ikkun god er Herreu mod Israel ( alle Troende ) , imod de Rette af Hjertet ( Djevelen , Verden og vort Kjod og Blod fetete nu sige , hvad de ville — Ps . 73 , 1 flg . ) . Noget Lignende sinder man ogfta andetsteds : I mit <b>Hjerte</b> vilde jeg esterteute , og min Aand ransctgede . Vil da Herren i al Evighed forkaste ' , og vil Han ikte mere

<h4>2610</h4>lig , mit Medmenneske , fta bliv ct Guds Barn ; vil du vere stor , riig og lykkelig , fta indtred i lesn l Christi Samfnnd ; vil dn vere , stjon , pregtig og anseelig , fta bliv et Tempel sor den Hellig Aand ; ler at foragte Verden med dens nsle og forgjengelige Stads og Pragt , og streb efter at forstaae o ^ erfare , hvad det Ord betyder : Vor Herres lesn Christi Naade og Guds <b>Kjerlighed</b> og den Hellig Aands Samfnnd vere med Eder Alle ! ( 2 Kor . 13 , 13 ) fta stal dn have Himmelen paa Jorden , ja i dig <b>selv</b> , i dit <b>Hjerte</b> . Christendommen er i iutet Stykke riugere end Verdett ; den har Wre , Hoibed , Herlighed , Glands , Skjotthed, Vellyst , Glede , Rigdom og Alt , blot medden Forskjel , at Verdens gode Ting fylde Oinene , Christendommen derimod Hjerterne ; Verden har mere Skin , Christend ommen mere Kraft ; Verden har Skyggen, Christendommeu Vesenet . Hvo som forgaber sig i den udvvrtes Pragt , som de fleste Christne tmtildags gjore , han forglemmer den indvortes og betenker ikke , at Sjelen af Gud er stabt til at vere Hans . Tempel , Haus Bolig , Hatts Himmel , som Han vil opfylde og lykftliggjote tned sit Lys , sin Klarhet » , Godhed og Salighed . § 33. Christend ommen medbringer et indvvrtes Guds Rige i Sjelen vg den Herres lesn og Hans Aands Forening med vor Aand . For Gnd er det ikkun om det Indvortes at gjore , og vor Christendom er ikke <b>sand</b> , saalenge som den Hellig Aand ikke beboer , regjerer og behersker vore Hjerter , - saalenge som Han ikke oplyser , vverbeviser , bervliger, gleder og styrker vote Sjele . Hos de Christne maa det hede : E e n G nd o g A l l e s F a d er , som er over Alle og ved Alle og i Eder Alle ( Eses . 4 , 6 ) . Vi hore Me Gnds Ord , foråt det blot stal lyde i vore Oren , men foråt det stal trenge ind i Hjertet med sin iboende guddommelige Kraft , og foråt den Hellig Aand , der er med og i Ordet , stat forene sig med vor Aand . Ligesom vi nu bor have dette for <b>Oie</b> i alle vore hellige Ovelser, vg ligesom vi Me maae staae os til Ro , sorinden vi fornemme Guds Rige i vor Sjel , saaledes kuuue vi ogfta sorst da rigtigt domme om hvilken Forskjel der er mellem Verden med dens Forfengelighed og Christendommen med dens Herlighed. Bliv en <b>sand</b> Christen , og lad Gud ftae sit Rige i dig ; fta stal du snart erfare , hvori der

<h4>2804</h4>Nogle af dem ligne Pattebomene , der endnu tigge oed Brystet og ikke gjore Andet end af og til at smile kjerligt og barnligt til sine Foreldre . Nogle ere ligesom de Born , der begynte at pludre og stamme , at sige Fader og <b>Moder</b> og at gaae ved Stole og Bente , — Nogle som de , der allerede tunne lobe og tale ferdigt , som blive holdte i Stole og underviste i Bon , Lesning og Skrivning , — Nogle som de , der allerede kunne hjelpe til i Huusholdningen, spare sine Foreldre sor at staae tidligt op og som vogte paa deres <b>Oine</b> , sor med storste Lydighed at ndfsre deres Befalinger . Nogle ligne de voxne Born , som ere vel vpdmgne og gjore sine Foreldre megen Glede med sin gode Opforsel . Men Allesammen ere de dog kjere Born , og <b>selv</b> det mindste af dem er Me fyr sin Skrobeligheds Skyld utelukket fra den faderlige <b>Kjerlighed</b> og Omsorg , ligesom <b>selv</b> det storste af dem ikke ved sit gode Forhold vinder noget Krav paa Faderens <b>Kjerlighed</b> , men tvertimod maa takke ham for Alt . Gud har befalet , at vi uaflateligen stulle bede vm vore Synders Tilgivelse , ligeftctvel som om vort daglige Brod . Vi leve Allesammen , Hoie og Lave , Strobelige og Sterke , Begyndende og Fnldkomne , af Hans Naade og Mistundhed , og der er Ingen af os , som har Noget at rose sig as nden af den barmhjertige Gnds <b>Kjerlighed</b> . § 52. Endelig maae vi 4 ) ogfta vide , at omendstjont det pacttigger os at efterjctge den sande Gudfrygtighed og vor Helliggjorelse med Flid og Iver , fta maae vi dog ikke sette vor Roes og Frimodighed for Gud og laugt mindre vor Retferdighed deri , tven alene i Sctmfnndet med Christo Jesu , sor hvis Skyld der ikke er nogen Fordommelse for os . I Jesu bloddryppeude Vunder have vi en aabnet Kitde imod Synd og imod Ureenhed, hvori vi daglig kntttte toe vvrt <b>Hjerte</b> ( Sak . ! 13 , 1 ) . Af Haus Fylde faae vi Alle og ! altid Naade over Naade ( Joh . 1 , 16 ) . Vi ! maae ikke opfttte Leren om den Netferdiggjorelse , j der steer ved Trven paa Christum , som om Christi For! tjeneste eengcmg gjorde os retferdige , og at vi siden setv ! maatte vedligeholde vor Retferdighed ved et helligt Liv og Levnet . Nei ingenlunde ! lesns Christus med sin Fortjeneste og sit Blod er alene og altid vor Retferdighed; udenfor Ham ere vi Intet , have Intet , fvrmace Intet ; ndenfor Ham er al vor Retferdighed

<h4>2946</h4>luvstidt Kappe , som man tun henger vm sig i Uveir vg forresten ikke stjotter om . Jeg er vis paa at det forekommer mange Mennester som eti ganske ligegyldig Sag og som et gammelt Eventyr , nåar man taler vm Guds store <b>Kjerlighed</b> til Mennestcstegteu, om Christi Blod vg Fortjeneste , om Troens Retferdighed , om Gudsfrygt , olli Dommedag , om Himmerige og Helvede . Jeg antager , at Mange <b>selv</b> maae indromme , at de ikke vide , hvad det Sprog betyder : Man trenger med Magt ind i Himmeriges Rige , og de , forn trenge ind med Magt , rive det til sig , - at de aldrig have hast nogen rigtig Sorg for sine Synders Skyld , ingen rigtig Bekymring for sin Salighed , - at de maastee aldrig have bedet og med Taarer cmmabt Gnd om et gjenfodt , bodferdig ! <b>Hjerte</b> , om <b>sand</b> Tro , <b>Kjerlighed</b> , Haab , o . dsl . De mene , at de allerede ere fta gode , som de kuuue blive , og at de Me behove at gjore sig gmae Håar for denne Sags Skyld ; te troe paa Christum ; derved blive de staamde ; man kan sige , hvad man vil , om den sande Troes Natur , om dens Frugter og Kjeudetegn , hvorved de ' stulle prove sin Tro , fta agte de det Me ; de. have ingen Tid til at granske og mnftge meget i aandelige Ting ; Ugen igjennem maae de fortsette sin Neringsvei, drive sin Handel , soge Vinding ; om Sondagen maae de efter at have hort paa en Prediken adsprede sig i lystigt Selstab , og saaledes gaacr det , indtil de doe ; i saadcm SMerhed vandre de fremad indtil de staae fyr Pvrten , som forer tem ud af Titeu ind i Evigheden . At , Gud , hvor jeg frygter , at Tanters Ord ville gaae i Opfyldelse paa stige Mennester ! Manget Menneske , siger han , troer at vere nok saa vel faren , vg nåar han kommer til Enden af Veien , saa <b>moder</b> den c v i g e D e d h a m . Nu , min kjereste Medchristm , lad hvem svm vil , sove , vere sikker og tåge sig det magetigt i aandelige og guddommelige Ting ; meu dig tilraaber jeg , som Englene lilmabie Loth : Skynd dig ! undfly for dit Livs Skyld ! ( 1 Mos . 19 , 17. 22 ) Bed , vaag , kjemp og strid , paa det du maa indgaae igjennem den stievre Pvrt og blive salig ! § 6. Men jeg maa her nedvendigviis besvare en Indvending , som Hyklerne pleie at sremfere . De sige : Van loerer ellers , at Christus alene har forhvervet vs Saligheten ved sit kvstbare Blod og sin Fortjeneste ; mm nu forlcmger man ogfta af os , at

<h4>3087</h4>aldeles baade mig <b>selv</b> og Dig og vandre fta lenge i fta stor SMerhed ? Ja , hvorledes har dog Du , min Gud , kunnet have fta lenge Taalmodighet » med mig ? Dog , Du er jo Gud og ikke et Menneske ( Hos . 11 , 9 ) . Hvor stor er Me din Mistundhed , min Gud ! Hvor stor er Me din Kjerlighed! hvor übegribelig er ikke din Viisdom , hvvr magelos din Kmft ! Jeg er et Under af din Godhed, Almagt og Viisdom . Min Fader ! Du har vel allevegue indtrykt din Kjerligheds Spor og merket <b>selv</b> den mindste Blomst tned din Finger ; men paa mig , paa mig seer man dog fremfor Alt , hvvr stor din Godhed er Hvad var jeg tilforn ? og hvad er jeg nu ? Tilforu eu Helvedbmnd , ett Vederstyggelighed og Gru for dine hellige <b>Oine</b> , Englenes Hjertesorg , Djevlenes Ven og Veninde , en Skamplet paa din Kirke , nu derimod formedelst din store Naade en Jesu Christi Brud , dit kjere Barn , Englenes Glede , Djevlenes Skrek , en Prydelse for din Menighed . Hvorledes kan jeg da andet, min Gud , min Fader , min Gjenloser , md prise Dig og din Godhed ? Du har gjort mig levende i Christo , da jeg var dod . Dn har udfriet mig af Helvede og ftt mig i Himmelen med Ham . Dn har forladt mig alle mine store , grove Synder og leget alle mine Brost . Du har draget mig op af Jordens Afgrunde , formere : min Storhet » og vendt om og trostet mig . Derfor vil jeg ogfta prise Dig og din <b>Sandhed</b> tned Harpespil , o min Gud ! jeg vil lovsynge Dig tit Citharm , Dn Israels Hellige ! Mine Leber stulle juble og min Sjel , som Dtt har forlost , lovsynge Dig . Lover Herrens Storhet » med mig , alle hellige Engle , alle gudftygtige Sjele , ja alle Skabninger , og lader os tilftmmm ophoie Hans Navn ! ( Ps . 71 , 22 flg . ; 34 , 4 ) § 10. I dette Lys . lerer dm ogfta rigtigt at kjende Gnd . De store og underlige Gjerninger , Gud har gjort med den , sammenholder den med Skriftens Udftgn og slutter af dennes Vidnesbyrd , som dens egen Erfaring forklarer , hvilket algvdt , kjerligt , alviist , almegtigt , majestetisk , übegribeligt Vesen Gud maa vere . Den trenger tned sit <b>Blik</b> ind i Guddommens Dyb og begriber efter sit Maal , hvad der er übegribeligt . Den stier ind i Gud ligesom i Solen og sinder Intet nden lutter Lys og <b>Kjerlighed</b> . Dm seer , hvorledes Hans <b>Hjerte</b> syder af evig Faderkjerlighed . Den stuer med Glede

<h4>3132</h4>ger og leder deres Sind til alt Godt , bevarer dem for dcmrlige , fattige og skadelige Fottagender , for SMerhed , Vildftrelse i Trom , for Synd og Ugudelighet»? Er det Me en stor Lyksalighed , at de Troende kjende stg <b>selv</b> og Dag for Dag lere sig <b>selv</b> bedre vg bedre at kjende , fta at de kuuue vise sig taknemmelige sor den Naade , som Gud fta rigeligen har anvendt paa dem , vogte stg sor sine endnu tilbageveteude Synder og med inderlige og andegtige Bonner verge sig derimod ? § 26. Er det ikke en stor Lyksalighed , at de kunne stue md i Guds Faderhjerte og dagligen forlyste sig ved Haus evige <b>Kjerlighed</b> og übegribelige Godhed , - at den Herre Jesus dagligen aabenbarer sig sor dem , viser dem sit broderlige , trofaste <b>Hjerte</b> og ei alene i Troen forener sig tned dem til Retferdighed og Salighed , men ogfta lader dem kjende denne sin salige Tilstand , fta at de ved Betmgtningen demf kunne fornemme Fred og Glede i dett Hellig Aand ? I Troens Begyndelse og saalenge den endnu er sped og ung , ere de Troende vel som umyndige Born , der eie en stor Arv , som de endnu ikke forstaae sig paa og saaledes hellerikke vide at glede sig i . Men de blive Me altid Bom ; de voxe i Trom og lere mere og mere at sorstctae dm Herlighed , som de have annammet formedelst Samfundet med Christo , og at stole og pukke paa demte Herlighed intod Djevelen og Verden . Er det ikke en stor Lyksalighed , - at de troende Sjele have deu Hellig Aand til en indvortes Lerer , der uttderviser dent saaledes i Guds Riges , Skriftens , Kvisets og Himmeriges Hemmeligheder , at de ftae en stor Vishet ) og Frimodighed i alle Ting og ere forsikrede om at Alt maa tjene dem tilgode , fordi de elste Gnd , - at de kunne merke Satans Tresthed og Underfundighet» og i sitt hellige Emfoldighed mode og modstace hans snedige Anlob , - at de vide rigtigt at vurdere vg stormodigt at foragte Verdens Forfengelighed, - at de <b>oine</b> Livet i Doden og overalt vise sig som Lys i Verdett ? Sandelig , dersom vi sik betmgte en <b>sand</b> Christen med opladte , aandelige og forklarede <b>Oine</b> , skulde vi see haus Hoved omgiret af et klart Skin og hans <b>Hjerte</b> glimrende og stmalmte langt mere end Ippersteprestens Bryststjold tned alle sine Wtelstene i den gamle Pagt . § ' 27. Saa prove nn Enhver sig efter denne Lere og undersoge , om han allerede har dette indre

<h4>3463</h4>Guds Side og de andre Troesovelser paa min Side * ) . 12. Denne Betragtning vil ogfta blive os til stor Nytte , nåar vi kvmme til at omhandle dm Brug , vi stnlle gjore af denne hoitrostelige Lere . Lader os imidlertid for OieblMet endnn overveie det Under af lesn Kjertighed , som ligger deri , at Han , vel vidende , at vor edle , udotelige Sjel Me kan tilfredsstilles og lykftliggjores med ttoget Andet end guddommelige , himmelske og evige Ting , fta og , at der hellerikke gives nvget bedre Legemiddel for vort Legeme , der sor Syndens Skyld er Doden underkastet, end det , som kommer fra Livsens Tre vg Gnds Pamdiis , at Han derfor har villet i sig <b>selv</b> gjore et kort Uddmg af alle guddommelige og himmelske Tiug og bestikke os sit hellige Legeme og kostbare Blod til en Mad og Drik , paa det at vi Alle i Ham knnde have Livet og al Fylde . Han er som en <b>Moder</b> , der giver <b>Die</b> og som Me tager den Mad og Drik , hvormed hnn neter og opholder sit spede Barn , noget andet Steds fra md . fra sig <b>selv</b> og fta at sige fra sit eget <b>Hjerte</b> og som meddeler Barnet sig <b>selv</b> , sit <b>Hjerte</b> vg sine Krefter , fta godt hun kan , ihvorvel jeg indrommer , at denne saavelsom alle andre Lignelser ere altfor ringe til at give et Billede af den Herres Jesu <b>Kjerlighed</b> i deu hellige Nadverd og af Hans hellige Legemes og Blods Samfund . Thi hvormed stal man sammenligne det , som Me har sin Lige ? Naar dm hellige Skrift vil beskrive os Gnds Godhed i ret sterke Udtrykke , fta taler den om Hans <b>Hjerte</b> . Ester dit <b>Hjerte</b> har Dn gjort alle disse store Ting , siger dm cmdegtige David . Gud siger <b>selv</b> : Jeg vil glede mig over dem , at gjore dem Godt , og jeg vil plante dem i dette Land i <b>Sandhed</b> , as mit ganske <b>Hjerte</b> og min ganske Sjel ( Jen 32 , 41 ) . Da dm hoiverdige Nadverd er et af Guds Kjerligheds storste Undere , knnde man afbilte den Herre lesnm , saaledes som Han sidder ved Bordet , uddeler det hellige Sakrament og rekker sine Disciple det velsignede Brod vg den velsignede Vim , tned den Overskrift : As ganske <b>Hjerte</b> og as ganske Sjel . Ak ,

<h4>3524</h4>ryggeslose Menneste til et Grempel for hans Medmennesker. Da han derfor den folgente Dag efter endt Prediken vilde trede frem med de Andre foråt annamme den hellige Nadverd , . lagde der sig en en saadan Taage for hans <b>Oine</b> , at han famlede omkring som en Blind og ikke kunde finde Alteret . Dette lagde de Andre Merke tit og vilde lede ham frem ; men han gik i sig <b>selv</b> og bekjendte , at han for Tiden endnn var uverdig til en saadan hellig Spise . Da han imidlertid kom ud af Kirken , blev han atter seende , omvendte sig siden til Gild af ganske <b>Hjerte</b> og nod derefter saligen det hoiverdige Sakrament. Hvad der af Guds synderlige Naade legemligviis vederfores dette Menneske til hans Fon bedring , det vederfares desverre Mange , som trede srem uden <b>sand</b> Auger og Omvendelse , tned et frekt vg sikkert <b>Hjerte</b> , aanteligviis til deres Forblindelse . Kommer dersor ikke til dette , hoihellige Sakrament ilden dybeste Idmyghed , uden omhyggelig Seloprovelse og uden Selvfornedrelse , I gudfrygtige Sjele ! Omendstjont den Herre Jesus er fremstillet for os til en Naadestol og lokker alle Mennester til sig med megen <b>Kjerlighed</b> og Vmlighed , saa maae vi dog i Betragtning af " vvrt Intet nenne os Ham med et ydmygt og bodferdigt <b>Hjerte</b> . Abraham kaldes en Guds Ven og nod den Wre , at Gud talte med ham som en Mand med sin Ven ; Herren vilde Intet gjore , for Han havde ladet sin Ven vide det ; men fe alligevel , tned hvilken dyb LErbodighed han pleiede at omgaaes sin Gud ; hau faldt paa sit Ausigt ; hau sagde : Se , Kjere , jeg har dristet mig tit at tale til Herren , uagtet jeg er Stov og Aste ( 1 Mos . 17 , 17 ; 18 , 27 ) . Jesus behandlede Hovedsmanden i Kapernanm med stor Vmlighed; saasnart denne gav Ham tilkjende , at hans Dreng laae verkbrndm og piintes svartigen , sagde Han strår : Jeg vil tomme og helbrede ham ; men Hovedsmanden svarer med stor Idmyghed : Herre , jeg er ikke verd , at Dn stal gaae ind nnder mit Tag ; men siig ikkun et Ord , saa bliver min Dreng helbredet ( Matth . 8 , 6. 7. 8 ) . Saaledes ogsaa : om end den Herre lesns siger : Jeg vil komme til dig og helbrede din Sjel , - jeg vil bespise dig med mit Legeme og leste dig med mit Blod , - jeg vil vere div , og du stal vere mitt , saa lader vs dog sige af Hjertet : Ak , min Herre Jesus ! hvo er jeg , at Dil kjender mig og agter

<h4>3668</h4>Sikkerhed kan behvlde dem i siv Magt ; derimod de Gndfrygtige og Udvalgte angriber han haardt , fordi hatt seer , at de ere undstyede hans Stritter og alvorligett forarbeide siv Saliggjorrelse . Du beflitter dig jo ogfta af Hjertet paa at efterfolge deu Herre lesum og daglig bere Hans Kors efter Ham ; Verdens Forfengelighed er dig en Vederstyggelighed og al dens Herlighed en Daarstab , hvilket atter vidner om at din Sjel er udkaaret af Gud til en storre Herlighed , efterdi Han har givet dm et saadant Sind , som itle setter nogen Priis paa det Timelige . Desnden kommer du jo daglig til div Frelser i en <b>sand</b> Omvendelse , i Tro , i Bon , i Hans Ords Horelse og Betragtning og i elt andegtig Brug af Haus hoihellige Kjerlighedstnaaltid . Du fremstiller dig jo dagligeu for Ham , ofrer Ham dit <b>Hjerte</b> og er rede til at fortere dig <b>selv</b> i Hans hellige Tjeneste . Altfta er du jo iblandt dem , som Haits himmelske Fader har givet Ham og solli Han ingenlunde vil stode ud ; det er : du hotet til Haus Udvalgte vg Elstede , Noget , som Han jo ogfta tilstilte beviser ; thi Hans Kmft opholder dig i al din Skrobelighed ; Hans <b>Oie</b> giver Raad paa dig ; Hans Haand bestjermer dig ; Hans Naade vederkveger dig ; Hans Aand troster dig . Hvad Mere stal deu misknntelige Gnd gjore somt forsikre dig om din Frelse ? § 31. Betenk saa for det Trede ogfta den t i l k o m m e n d e T id , og hvor hoit og dyrt dilt Fader har lovet , at Han ingenlunde vil slippe dig og ingenlunde forlade dig , " - at Ingen stal rive dig ud af Hans Haand og Intet stille dig fra Hans <b>Kjerlighed</b> . Han ftae vel , at du itle " tuude blive fri for Anfegtelser , men vilde komme i Ild og Vand ; men dersor har Han ogfta forjettet , at nåar dit gaacr igjennem Ilden , stal dn itle brendes og Lnm ikke fortere dig , og nåar dn gaacr igjcnn m Baudene og igjennem Flotene , stulle de ikke overskylle dig . Nåar du for Tiden er vis paa at dit horer tit de udvalgte Guds Born , fta tan du jo med al Net hentyde paa dig og af Hjertet tilegne dig alle Gnds Forjettelser , hvori Han har lovet at opholde sine Egne i Troen indtil Enden og ved sin Almagt bevare dem til Salighed og af hvilke vi ovenfor have anfort de fornemste . Betent ber ogfta disse : Det Lidet , Vreden vårede , skjulte jeg mit Ansigt et Oieblit for dig ; men jeg har for-

<h4>3976</h4>Lignelser , fornemmelig tagne af Foreldrene og deres <b>Kjerlighed</b> , somt tilkjendegive , at man i Bomenes Opdmgelse ikke blot stat see paa Mennester , men forst og fremst paa den , fm hvem de kjodelige Foreldre have sin <b>Kjerlighed</b> , sin Fornme , sit Format » og Alt . Ligesom et Menneskes Dage folge paa hinanden og fta at sige danne en Kjede , saaledes folge ogfta Foreldrenes og iser Gnds Velgjerninger paa hinanden , fta at man kunde kalde et gudfrygtigt Menneskenes Liv en gylden Kjede af Guds <b>Kjerlighed</b> og Velgjerninger eller eu Vev , hvori Rendingm er idel Guds Naade og Mistundhed og Isletten idel Guds Laugmodighed , Barmhjertighed og Forsorg . Det er eu Lysthauge , der overalt pruuker med Guds Kjerligheds Blomster , som voxe i Hobetal midt iblandt Tornene ( de Trengsler , Anfegtelser og Farer , som mode os ) . Skriften vidner , at uaar vi blive asvenuede fm vor Moders Bryst , blive vi dog heugmte ved vor Guds Kjerligheds Bryst , som vi ei eugang i vor hoie Alderdom kunne nndvree . Herren siger <b>selv</b> : Jeg , jeg lerte Esraimat gaae ; jeg tog dem paa mine Arme . Jeg drog dem med Kjerligheds Reb ( Hos . 11 , 3. ' 4 ) . Moses siger til " det ismlitiste Folk : Du har seet , hvvrledes Herren din Gnd har baaret dig , ligesom en Mand berer sin Son , paa al den Vei , I have vandret ( 5 Mos . 1 ) 31 ) . Dette er et vidunderlig deiligt Billede , hentet fra en Fader , der enten hjemme af <b>Kjerlighed</b> eller paa Velelt af Medlidenhet ) og i Betragtning af Barnets Svaghed tager sin Son paa sine Arme eller paa Ryggen og berer ham foråt bringe ham afsted mcd sig . Ak , algote Gud ! hvvr fodt er ikke dit Ord for dem , som annamme det med et troende og andegtigt <b>Hjerte</b> ! hvor trostelig er itle din <b>Kjerlighed</b> , som Du fta sodeligen beskriver , for dem , der betragte dm med et taknemmeligt <b>Hjerte</b> ! § 15. Skriften stger ettdvidere : Herrens <b>Oine</b> ere vendte til de Retferdige og Hans Oren til deres Raab ( Ps . 34 , 16 ) , - ret ligesom en Fader og <b>Moder</b> , <b>selv</b> om de ere sysselsatte med andre Ting , dog stedse holde <b>Oie</b> med det omspringendc og legende Barn , fta at det ikke kommer til Skade ; dersom det snubler og striger , lobe de hurtigt til , og der er Intet , som for bringer Moderen paa Benene end hendes kjere Barns Skrig . Den

<h4>4158</h4>stode udeufor og begjerede at tale med Dig ( Matth . 12 , 49 ) . Iligemaade vil ogfta jeg , mitt Forloser , uaar der blioer Sporgsmaal om hvem der er mig deu Kjereste , pege paa Dig , ftaledes som Du heuger paa Korset , og sige : Se , min <b>Hustru</b> , mit Barn , min Fader , min <b>Moder</b> , min Broder og mit Alt ! En as dine Blodvidner har sagt , at han elskede sin <b>Hustru</b> og sine Born fta hoit , at om hele Verden var suld af Guld og tilhorte ham , vilde han gjerne hengive den for at kuune have dem hos sig i sit Fengsel og dele sit Vand og Brod med dem ; men alligevel elstede han dog den himmelste <b>Sandhed</b> og Ehnstus endnu hoicre . Ret fta ! Lad Lyseue brende , medens det er Nat ; men nåar Solen staaer op , behover man dem ikke lenger ; da slukker man tem gjerne . <b>Hustru</b> , Bant , Fader , <b>Moder</b> ere kutt smaae Lys ; men Du , Herre Jesus , er miv Sjels Tol ; jeg tan letteligen uudvere Lyseue, uaar jeg tun har Dig . § 12. Jeg Elster Dig , Herre Jesus , hoiere end mine <b>Oine</b> ; thi om jeg ogfta stulde tabe dem , fta har Du dog tovet , at Du vil lede mig tned dit <b>Oie</b> , som Du desuden gjor , <b>selv</b> om jeg ogfta har <b>Oine</b> ; Du er miue Oines Lys , Glede og Trost ; hvor jeg vender mine <b>Oine</b> hen , seer jeg intet Andet end Forfengelighed ; men nåar jeg seer paa Dig , h ' ar min Sjel Nok , — hoiere end mit <b>Hjerte</b> ; jeg kan itle sige , hvor jeg hater mit <b>Hjerte</b> , nåar jeg betmgter det udeusor Dig ; thi hvad er det as Naturen Andet end en Verden as Uretserdighed og Ondstab ? betmgter jeg dct derimod , ftaledes som det ligger for dine Fodder , bestenket med dit Btod , og som det begynder at lenges ester Dig og din <b>Kjerlighed</b> , da er det mig kjetl , ikke at det er mit , men at det er dit ; mit <b>Hjerte</b> er en Tavle , hvorpaa findes et Udkast og Aftids as dit Billede, min korsfestede Frelser ; Tavlen stjotter jeg ikke om ; kun sor det edle Billedes Skyld sinder jeg Behag i den ; og dog , om jeg ogfta stulde tabe mit <b>Hjerte</b> , vilde det vere tnig ligegvldigt , efterdi du er og bliver mit <b>Hjerte</b> , — hoiere end mit Liv ; thi udeu Dig , som er mit rette og sande Liv , er det mig til ingen Nytte , — hoiere end min Sjel ; denne cr mit dyrebareste Klenodie , som Dn veed at jeg setter over Alt , hvad der er i Verden ; ilkedestomindte , dersom det var muligt , at jeg kunde

<h4>4344</h4>almegtige , alvise , allestedsnerverende og hoilovlige Gud , mitt Skader og Opholder , min Frelser og Talsmand , maa vorde hoiligen eret og priset . § 35. Medens jeg skriver dette , motlager jeg sta ett hoi og kjer Haand en liden Bog , hvori en gitdfrygtig <b>Kvinde</b> , udfpnmgcu af ftruem Wt , har plantet mange stige Blomster til Guds Wre , hvilke have veret mig- <b>selv</b> fta behagelige og gledelige , at jeg sikkert troer , at de ogfta ville finde alle andre med hellig Begjerlighed opstldte SjeleS Bifald. Ak , siger hnn . Du deu evige Faders evige Ord , giv Acmd og Ord til at elste div Priis og prise din <b>Kjerlighed</b> ! ftruud , allerhoilovligste Frelser, Dig til evig Wre , at din Priis maa vere mit Lio og mit Liv div stcdscvarmde Priis ! Hcrte Jesus , " lad Alt , hvad jeg teuker og gjor , vete gjort i og af Dig til div Tjmeste og ucudelige Heder ! Ak , kuute jeg tjeue Dig alene med alle Mennesters Sandser og Evner ! ja turde jeg utlede mig alle Euglcs Kraft , at jeg dermed kunde lovsynge Dig lifligt paa Wrte-Engles Viis ! Ak , almegtige Jesus , , som har givet tnig ftamegen Attråa , giv mig om endog kun den mindste Evne til at iverksette dm ! Om alle vore Stille og Blodsdmaber bleve til Engle og Serafer , knnde de dog aldrig ftlldtatle Dig , Herre Jesus ! Om vi knnde oposte tnsind Liv derfor , hvad var det vel for dett , fom opfylder Evigheden ? Dog stal man ikke tie , men forkvnde Gnds Priis . Derfor siger jeg : Dig vere Lov ! Dig vere Priis ! Dig vete Wre ! Dig vere Heder , Magt , Kraft , Styrke , Sang , Trinmf , Jubel og Herlighed i Tid og Evighed ! ofv . Jeg vil vete umettelig paa din Wre , efterdi Dtt har veret umettelig paa min Salighed . Ak , tnmtte min Mund aldrig rore sig uden til din Priis ! maatte mine <b>Oine</b> altid bevege sig ' til din Wre ; ja maatte Pnls og <b>Hjerte</b> vere lutter Harper og Psaltere til derpaa at lovprise din ulidelige Angest ! Jeg gleder mig fornemmelig til Evigheden , at jeg der stal ftae Tid og intet Andet have at gjore end at prise Dig uden Ovhon Var det tilladt at afstjere dette Livs Tmad , fta sonderrev jeg dm idag , for strax atkomme did og retteligm der prise Dig . Ak , evige Forloser ! los snart de Baand , som hindre min Aand fra at opsvinge sig til Din Priis ! Dog hellerikke heri , forn jeg vil , men som

<h4>4415</h4>§ 5 . For det Tredie betragte vi hende ogfta som et Levnets-Speil . Huu legger sig hm til den Herres lesn Fodder som et levende Offer , og da hun ei alene griber og kysser dem , men ogsaa veder dem med sine Taarer , torter dem med " sit Håar og salver dem med Balftm , fta tilkjendcgivcr hnn udeutvivt hermed , at hun nn vil hengive sig <b>selv</b> med Alt , hvad hun har og formaaer , til Herrens Tjeneste , - at hun ikke lenger vil vere sin Egen , mett fremstille sig for sin Frelser som Hans Eiendom. Af Syndens Redstabcr gjor hnn Troms og <b>Kjerlighedens</b> Redstaben Siv Muud , som hnn hidtil havde frembudt til utugtigc Kys og Elskere , trykker hun nu til sin Meglers Fodder og helliger Ham dm. Sine <b>Oine</b> , der hidtil havde veret sande Lygtemend , anteudte af Djevelen til at forfore Ungdommen , ere uu blevue Taarckilten H eudes Håar , der hidtil havde veret sande Satans Garn , gjor huu til den Herres Jesu Fodtlud . Siv Salve , hvormed hun hidtil havde givet sit utyske Legeme et smutt Udsemdc , udoser hun uu villigen over Herrens Fodder . Kort , hun forpligter sig hermed til i al Evighed at vere og blive den Herres Jesu <b>Elskerinde</b> og Tjmerinte . Huu attmaer iutet Andet end at vere et Redstab for Hans hellige Villie ; huu ofrer og hengiver sig til Ham med Alt , hvad hnn er og har , og onster at fortere sig <b>selv</b> i Hans hellige Tjeneste . Dette er nu en uudeblivelig Frugt af den sande Tro og et übcdmgctigt Kjentetegn paa samme , uemlig uaar Meuuestet tmddmger sit <b>Hjerte</b> sra Verden og overantvordcr det til deu Herre lesum , - nåar det bliver eu hellig Forandring våer hos sig , - nåar det er rede til at opofre Alt , hvad det har og formaaer , ja <b>selv</b> sit Kjereste og Bedste paa <b>Kjerligheden</b> til lesum , - nåar det med et villigt <b>Hjerte</b> hengiver sig til den hellige Lydighed imod Ham og inderligt

<h4>4556</h4>ger ved hendes Bryst eller , leger omkring hende ; meit fordi man er vant dertil , bliver det ikke cmseci for noget Synderlig ! ; dersom imidlertid Mcmdm blioer dodeligcn syg eller kommer i nogen anden Fare og Nod , - dersom Batnet falder ftrligt eller bliver angtebet af et Dyr , da begynder <b>Kjerligheden</b> at udbtede sig og at gjore Underverker ; da engstes det kjerlige <b>Hjerte</b> ; da vaager , da beder, da greder , da snkker hun , da loder hun til , da glemmer hun stg <b>selv</b> , sin egen Pleie , Magelighed , Sundhet » , ja sit eget Liv og er rede til at oposte og heugive sig <b>selv</b> for dett Elskedes Reduiug . Naar uu den uaturlige <b>Kjerlighed</b> virker dette , ester hvad vi daglig see for vote <b>Oine</b> , hvad maa da ikke den gudelige og himmelste <b>Kjerlighed</b> - udrette ! De troeude Sjele gaae ofte der fta übetydelige og eenfoldige, at det hverken falder dem <b>selv</b> eller Andre ind , at de skalte uere uogeu synderlig Nidkjerhed for Gud ; tuen uaar eu Leilighet ) viser sig , og de see , at deres Guds og den Herres lesn LEre cmgribcs af Satan og den onde Verten , eller nåar Nestens Tarv fordrer det , da bryter deres <b>Kjerlighed</b> ud med Velde ; dc glemme sig <b>selv</b> og ivre for Gud og Nesten , dct maa nn forresten gaae , som det gaac vil . Saaledes er da den gudelige Nidkjerhed Noget , som nodvendigviis folger af Troen , der forener os med Christo . ' Ligesom det er en Moders og trofast LEgtefelles Art at elste med Nidkjerhed, ftaledes er dot ogfta den troende Sjels Art . Hiint kommcr og ndspringer af Naturen , dette af Naaden , og ligesom den <b>Hustru</b> , den <b>Moder</b> , som ikke ittderligt elsker sin Mand , - sit Barn og ikke i Nodstilfelde er nidkjer sor dem , itle cr at agte for nogen trofast Wgtcfelle og retstaffen Moder, ftaledes kan hellerikke den , som Me elster Gud inderligt og itle vcd given , Anledning er nidkjer for Hans LEre , anstes for troende og retsindig . § 10. Men lader os uoget yderligere betragte og oplyst , hvad der oveufor ( § 7 ) er sagt om dm gudelige Nidkjerhed ; fta vil ogfta famnes Nodvcndighed bedre indsees . Den troende Sjel cr nidkjer for Gnd og Alt , hvad der er gnddommeligt. Efterdi dm ei aleue af Hans Ord , meu ogfta af egen Erfaring veed , hvor kjerlig , mistnntelig , naadig og barmhjertig Gnd er , - hvor godt han mener det med Menneskene , - hvor hoit Han elsker tem , og hvor troligm Han sorger for

<h4>4952</h4>troste en Bedrovet , meddele , en Bekymret Raad , bespist en Hnugrig , leste en Torstig , vederkvege en Syg , formane en Synder oso . De ere ligesom en <b>Moder</b> , hvis Bryster Melken trenger og som lenges efter sit eget eller et andet Barn , som kan nyde den . De ere ligesom et Tre , der staaer suldt af Frugt og boer sine Grene mod Jorden , ftm om det vilde byde Meunestme dem ; kunde det tale , vilde det ganske vist sige , at det somt adlyde sin Skaler villigen og gladeligm rakte Menneskene dem . De troende og tjerlighedsfulde Sjele ete ligesom de Kar paa et knnstigt Vandspring , som af en ovenfor staaende Figur eller ogfta ofte gjennem stjulte Gange fyldes ftaledes , da de allevegne flyte over og fylde andre nedenfor staaende Kar eller vande den hele Hauge . De trostes af Gud og troste igjen Andre med den Trost , hvormed de <b>selv</b> ere blevne trostede ( 2 Kor . 1 , 4 ) . Guds <b>Kjerlighed</b> er udost i deres Hjerter , og disse stromme over af Glede og meddele Andre , hvad de <b>selv</b> have annammet , hvorom Apostelen har det merkelige Udftgn : Men Eder forfremme og overflodiggjore Herren i <b>Kjerligheden</b> til hverandre og til Alle ( erstatte alle Eders Mangler efter sin Mistundheds Rigdom og give , at I maae stromme over af Broderkjerlighed ) , ligesom og vi have den til Eder ( 7 r ^ , kci ^ « < 7 « e itttt Tre ^ lcrl / k ^ cr ^ i 5 ^ « / « ^ c / 3 « ) . ) . ^ ) . o ? > c x « i e ^ ? r « ^ s « s — 2 Thess . 3 , 12 ) ! § 20. Broderkjerligheden er 2. ) hjertelig og oprigtig . Thi esterdi den Herre Jesus boer i Hjertet og opfylder det med sin <b>Kjerlighed</b> og Naade og med sin Aand , fta fordeler <b>Kjerligheden</b> sig derfra som en reen Kilde i det ganske Menneste og gjor det kjerligt og venneselt , ikte blot efter Skinnet, men i <b>Sandhed</b> , ikke blot udvortes , men ogfta indvortes . En ftnd Christen har et kjerligt <b>Hjerte</b> , en livsalig og trosteriig Mund , et laarefuldt , medlidende <b>Oie</b> , et aabent , villigt Ore , en gavmild Haand og hurtig Fot » . <b>Kjerligheden</b> er en himmelst Lue , der , cmtcmdt i Hjertet , udbreter sig overalt og staaer ud gjennem Menneskets <b>Oine</b> , Oren og Mund . Herom siger Apostelen : Lader os vere <b>Sandheden</b> troe i <b>Kjerlighed</b> ( lader os bestrebe os for at vor <b>Kjerlighed</b> maa vere ftnd ) , som <b>Sandhed</b> er i Jesu ( ligesom Hans <b>Kjerlighed</b> var lutter <b>Sandhed</b> , - ligesom Han var fuld af Naade og <b>Sandhed</b> , ftaledes maa det ogfta vere

<h4>5116</h4>eller den vil rede hans Seng , eller den vil tale trosteligt og medlidende med ham eller gaae ud og blive hans Talsmand hos Andre . En vis fattig <b>Kone</b> , hvis Boru <b>selv</b> sogte Brotet ved Andres Dore , havde et Par Hons , som huu emerede med de Smuler , der bleve tilovers fra hende og hendes Bom ; denne <b>Kone</b> havde i sit Nabolag et sygt og forladt Menneste , til hvem hun jevnligen kogte et af de 3 Eg , som hendes Hons lagde , og bragte ham det med et kjerligt <b>Hjerte</b> . Troe I ikke , at dette var en kostelig Almisse i Guds <b>Oine</b> ? § 47. Lader os endelig gjore Godt imod de Fattige med Forglemmelse as os <b>selv</b> , uden Tmgten efter egen Roes eller forfengelig 3 Ere ! Bekjendt er det , hvad vvr Frelser siger herom : Nåar du gjor Almisse , saa lad diu venstre Haand ikke vide , hvad din hoire gjor , paa det din Almisse kan vere i Londom , og din Fader , som seer i Londom , Han skal betale dig aabenbare ( Matth . 6 , 3. 4 ) . Almisserne sammenlignes , som vi ovenfor sagde , med Seden eller Sactkomet ; dette maa ikke blive liggende ovenvaa Ågeren — thi da vil det blive opedt as Fuglene — , men bedekkes tned Jord , ftafremt det stal bere Frugt ; derfor er Latidmcmten ogfta strax ferdig tned sin Harv og ster Intet heller end at al Seden kommer godt ned . Saaledes maa det ogfta vere med Almisferne . En christelig Sjel lader stg noie med at vide , at <b>selv</b> det Mindste , som han gjor i <b>Kjerlighed</b> , bliver optegnet i Guds Bog , hvorvel han itle anseer sig verdig dertil ; han veed godt , at han endnu gjor meget sor Lidet , og at <b>selv</b> om han gjor Alt , hvad han bor gjore , er og bliver han dog en unyttig Tjener , som ikke har Andet end Gnds Naade og den korsfestede lesum at rose sig af , ost . Gud give os et saadant <b>Hjerte</b> ved Christnm lesum i den Hellig Aands Kraft ! Ham vere Lov og Tak i al Evighed ! Amen .

<h4>5520</h4>vi ere Alle Fremmede og Gjester i Haus store Verdmshnus , og Han kan lade os beverte og betjene, som Han vil ; mm vi vide ogfta , at Han heri gaacr frem med den storste Viisdom , Retferdighed, Godhed og <b>Kjerlighed</b> ; alle Hans Stier ere Mistundhed og <b>Sandhed</b> , og hvad Han forordner og bestikker , det er Altsammen herligt og priseligt . Han beskrives os ikke blot som en Herre , " der i " sit eget Hnus handler efter sit Godtbesiutende , meu tillige som en Fader , der ftrger for sine Born med stor <b>Kjerlighed</b> og Troftsthed . Skulde vi da ikke gjeme lade os noie med den Stilling , hvori Han har sat os ? Sknlde vi knurre imod Ham og modsige Ham ? Hvad kan det vel hjelpe ? ville vi ikke blot paadmge os Hans Vrede og Unaate ? Skulde vi fordriste os til at gjore Indvmdiuger mod Hans alvise Forsyn og Regjering ? Ete vi da visere end Han ? eller ftrstaae vi bedre , hvad der tjener til vort eget og Andres Vel , end Han ? eller kuuue vi styre og sorftrge os <b>selv</b> bedre eud Han ? Dersom Valget mellem Lykke og Ulykke , Rigdom og Fattigdom , Overflod og Mangel var lagt i vor Haand , bnrte vi da ikke , som lydige og elskelige Born , overlade og henstille det til Ham , som vor Fader , og bede Ham , at Han vilde velge for os , efterdi vi itle forstaae vort eget Bedste og ikke knnne <b>oine</b> Udftltet ? Mm siden Han nu har valgt ftr os og tilskikket os den Medgaug eller Motgang , hvori vi besinde os , fta er det vist , at Han ikke har sundet noget Andet tjenligt til vort Bedste . Sknlde vi da ikke vere tilfredse med Hans Valg ? § 34. Lader os desuden ofte betenke , at Gud har begavet os fta rigeligen med aandelige og himmelske gode Ting i Christo lesn , sin tjere Sov , og givet os saadanne Forjettelser mcd Hensyn til de timelige , som vi nok knnne forlade os paa , og ftaledes leve tilfredse . Han har jo skjenket os sin allerkjereste Son tilligemed al sin Naade , Syndernes Forladelse , den sonlige Udkaarelses Herlighed , den Hellig Aands Trost og Udsigten til den himmelste og evige Arvedeel ; stulde vi da gjore os store Bekymriuger for det Forgjengelige , der intet kan hjelpe vore Sjele , ret som om den Gud , der har skjenket os sin Son , sit <b>Hjerte</b> og sin Himmel , ikke ogfta kunde og vilde forsyne os tned nodtorftigt Underhold her i Verden , eller som om vi itle kunde vere lykkelige nden Overstodighed paa timeligt

<h4>5592</h4>Naar vi i Kirken hore en aaudrig og nidkjer Prediken eller annamme den hellige Nadverd , tyktes det vs ofte , som om vort <b>Hjerte</b> er fuldt af Tro , Andagt, <b>Kjerlighed</b> og Iver ; men nåar vi atter komme ud i Verdeu , taler Alt sig snart igjen . Oste fremtrede vi ogfta for Gud , som om vi vare ganske rene og plctfrie ; vi troe , ftm Fariseeren , at vi itte have Grund til at bede Gud om Noget eller afbede Noget hos Ham , men kun til at tatte Ham ; vi finde det paaftldmde , at Gild vil gjore nogen Fordring paa os , uagtet vort <b>Hjerte</b> indbilder sig , at vi have gjort Alt , hvad vi vare skyldige at gjore . Dette er uu det menneskelige <b>Hjertes</b> Beskaffenhet ) , saalenge ftm det forbliver overladt til sig <b>selv</b> . Nåar det derimod bliver fvrnyet ved Ehristum og formedelst Troen forenet med Ham , begynder det af al Kmft at strebe efter gudelig Oprigtighed og Retsiudighed og at beflitte sig paa Scmddruhet» imod sig <b>selv</b> , imod Gud og Nesteu , som vi kuuue see af de Ord hos den hellige Apostel , som vi have vplest og efter hvis Foranledning vi da nu stulle omhandle den troende Sjels Oprigtighed og Retsindighed . Gud give , at det maa stee med megeil Frugt og Nytte , for lesn Christi Skyld ! Amen . § 6. Til de Herligheden som vor Sjel for Syndeftldet var begavet med af Gud , horte ogfta , at den i alle Dele var reen og uden Falskhed . Den var , sym vi i Begyndelsen af dette Verk have sagt , et blankt Speil , hvori det evige Lys gjenstmalede sin rene Glands ; den var som et stebet Kry sta lg las , fnldt af reent Vand , hvorigjmuem Solen skinner ; der var intet syndigt Dynd , ingen Plet , ingen Ureenhed at opdage deri . Mm ved Synden er dette rene Vand blevet mudret , det blaute Speil plettet og det klare Glas opsyldt med Morke ; jeg mener : Sjelen har forudm andre Gaver vgfta mistet sin Retsindighed og Sanddruhet » . Sactsnart de sorste Mennesker vare ftldne , skjulte de sig for Gud Herreus Ausigt mellem Hauge us Treer ( 1 Mos . 3 , 8 ) ; de vilde holde sig stjulte for delts <b>Oine</b> , for ' hvem Intet er stjnlt ; de begyndte at undstylde sig og at kaste Skylden for Overtredelsen den Ene paa den Anden ; ja de havde endog gjerne lagt den paa Gud <b>selv</b> , idet Adam sagde : <b>Kvinden</b> , som Du gav mig til at vere hos mig , huu gav mig af Treet ,

<h4>5605</h4>5 , 8 ; 10 , 16 ; 18 , 2. 3 ) . Der var Ingen , som Han med storre Alvor og Nidkjerhed modfttte sig , eud Hyklerne ^ der soge at bedrage Gud og Meunester med et falskt Skin . De hellige Apostle minde oste om at vi skulle vandre i <b>Sandhed</b> ( 2 Joh . 1 , 4 ; 1 Joh . 1 , 6. 8 ) , . at vi stnlle vere <b>Sandheden</b> troe i <b>Kjerlighed</b> og vide , at der er <b>Sandhed</b> i Jesu ; vi stulle ifore os det uye Meuneske , som er stabt efter Gud i Saudhedens Retferdighed og . Hellighed ( Efes . 4 , 15. 21. 24 ) ; vi stulle vere rene og uden Anstod indtil Christi Dag ( Filipp . 1 , 10 ) ; vi stulle have et oprigtigt S ' md ( 2 Pet . 3 , 1 ) ; de onste , at Fredens Gnd vil hellige os ganske og aldeles , og at vor ganske Aand ( i alle Dele , i Tro , <b>Kjerlighed</b> , Haab , Ord og Gjerninger ) tned Sjel og Legeme maa bevares ustraffelig i vor Herres Jesu Christi Tilkommelse ( 1 Thess . 5 , 23 ) . § 11. Ligeledes have vi alle hellige Guds Boms Exempler for os , idet de stedse have beflittet sig paa at have et oprigtigt Hjertelag imod Gud vg Meunesten Om No ah siges , at han var en retferdig og ustraffelig Maud blaudt sine Samtidige og vandrede med Gud ( 1 Mos . 6 , 9 ) . Herren <b>selv</b> giver Job det Vidnesbvrd , at han var ustraffelig og retskaffen , gudfrygtig og veg fra det Onde ( Job 1 , 1. 8 ; 2 , 3 ) , ligesom Han ogfta giver David det Vidnesbyrd, at hau vandrede for Ham med et fnldkomment <b>Hjerte</b> og med Oprigtighed ( 1 Kong . 9 , 4 ) . Han tiltaler ogfta sin Gut ^ med disse Ord : Miv Gud ! jeg veed , at Du prover Hjerter og har Velbehag i Oprigtig h ed ; derfor har jeg i mit <b>Hjertes</b> Oprigtighed gioet atle disse Ting ( som jeg har samlet til Tempelbygningen ) fr i v i l l i g e n ( 1 Kron . 30 , 17 ) , og paa et andet Sted : Lad Ustraffe lighed og Oprigtighed bevare mig ! ( Ps . 25 , 21 ) Da Kong Ezekias hjemsogtes af Herren med en dodelig Sygdom , knnde han bemabe sig paa at han havde vandret troligen for Gud og med et retstaffent <b>Hjerte</b> og- gjort det , som var godt for Hans <b>Oine</b> ( Efti . 38 , 3 ) . Denne Dyd roses ogfta hos de. to LEgtefolk Zakarias og Elisabeth, idet der siges om dem , at de begge vare retferdige fyr Gud og vaudrede ustraffelige

<h4>5622</h4>deres venstre Haand ikke stal vide , hvad den hoire gjor ; de vide godt , at de ingen anden Wre derfor kunne gjore Fordring paa end den , som tilkommer en Tjener , der berer sin Herres Pung og paa hans Befaling giver en Fattig en Daler ; de give emftldeligen , ikke for at hoste Wre og Roes af Verden , itte med hovmodig ! <b>Hjerte</b> og Foragt for de Fattige , men med et ' reent Hensyn til Nestens Nod og Gnds Bud . § 16. Saaledes forholde de sig ogfta i alle andre Ting , som de ifolge sit Kald og sin Stilling have at forrette . Ete de faldede til det hellige Predike-Embede , fta bestrebe de sig efter Apostelens Exempel ftr vcd Guds Naade at forrette Alt i Emfoldighed og gudelig Retsindighed . De forfaifle ikke Guds Ord ( med Mennestesnak , kjotelig Sands , verdslig Viisdom , Streben efter egen Wre og Notte ) , men frembere det i Retsindighed ; de tale som es Gud , for Guds Aasyn , i Christo ; de anbefale sig ved <b>Sandhedens</b> Aabenbarelse til alle Menneskers Samvittighed for Guds Aasyn ( 2 Kor . 2 , 17 ; 4 , 2 ) . Retsmdige Prester ere som Duerue , der samle det reneste Korn i stn Kraa og siden med al Magt opstode og udtomme det i sine Ungers Mund ; hvad de have lert af Guds Aand og Naade , - hvad de have samlet af Guds Ord , - hvad de have suget af Christi Vunder , - hvad de have iagttaget ved egm Erfaring , det maae de med brendmde <b>Kjerlighed</b> udsse og indgyde i Tilhoremes Hjerter , ikke foråt vinde egen Wre og Fordeel , ikke foråt tekkes Mennester ( nemlig forn Mennester , ester Kjodets Sands ) , men fomt teklcs Gud og befordre Menneskenes Salighed . De maae . itle behage sig <b>selv</b> , mm me og alene soge at vere Gud og folgelig ogsaa Nesten til Behag i det Gode , til Opbyggelse ( Rom . 15 , 1. 2 ) . Saaledes har ogfta en christelig Ovrighed itte sine egen Wre , Magt , Rigdom , Vellyst og Magelighed for <b>Oie</b> , men forst og fremst Guds Wre og det almene Vel . Den veed , at den har al sin Wre og Magt sta Gud og for Gud , og at Magten ikke er den given til at stade og otelegge , men til at forbedre og bevare . Den betmgter itte Underftcttterne med stolte <b>Oine</b> og Foragt , men med et faderlig! og kjerligt <b>Hjerte</b> og beflitter sig paa at vogte dem med al Trostab og regjere dem med

<h4>5679</h4>z 39. En <b>sand</b> Ehristen veed 4 ) godt , at Troen og den egle Christendom er en bestandig Kamp og Strid , og at man dagligen maa efterfolge sin Frelser i Verdenssorftgelse , i Selvfornegtelse og i villig Paatagelse af Korset . Derfor sparer han ingen Moie ; han faster , beder , strider og kjemper hver Dag ; han endser ikke , om han derved votder sig allehaande Uleiligheder ; han vil gjerne give Slip paa sin Magelighed og underkaste sig alstens Besoerligheder , nåar han blot kav tjene Gud og sin Neste og forarbeide sin Saliggjsrelse. En Hykler derimod er en Fiende af Christi Kors ; han veed ikke , hvad det vil sige at ftrftge Vcrden , at ftrnegte sig <b>selv</b> og at tåge Christi Kors paa sig og efterfolge Ham . Han veed itte af nogen Kjodets Spegelst og Undertvingelse ; det vilde itte vere ham tilvas , om han stulde afbryde sin Sovn og cmomde nogen Tid as Natten til Bon og andre Gudfrygtighedsovelser , og endnu mindre , om han stulde have nogen Gjenvordighet » , Uro eller Trengsel for Chiistendommms Skyld . Han onster at tomme til Himmelen i godt Mag , uden Vaagen , Bon , Faste , Kamp og Strid ; det maa itte falde ham suml ; hau gider hellerikke tenke meget over hvorfor vor Frelser har sagt : Streber alvorligen at indgaae igjennem den sneore Port ! ( Luk . 13 , 24 ) og Paulus : Arbeider paa Eders egen Saliggjorelse med Frygt og Bevelse ! ( Filipp . 2 , ' 12 ) og paa et andet Sted : O Guds Me nu eske , jag efter Retferdighed, Gudsfrygt , Tro , <b>Kjerlighed</b> , Taalmodighed, Sagtmodighed ; strid Troeus gode Strid ; grib det evige Liv , til hvilket du er kaldet ! ( 1 Tim . 6 , 11. 12 ) , § 40. En <b>sand</b> Ehristen er 5 ) ydmyg og ringe i sine egne <b>Oine</b> . Han troer itle , at han staaer over nogen af sine Medbrodre ; han betmgter alle sine Gjerninger ligesom gjennem en omvendt Kikkert , og de er / smaae og langt bortfjernede fm hans <b>Oine</b> . Naar han tilligemed Andre gaacr til den hellige Nadverd , fta holder han med oprigtigt <b>Hjerte</b> for , at Ingen er nverdigere end han <b>selv</b> . Han er vel tilfreds , nåar han bliver foragtet af Andre , fordi han foler sig forvisset om at han godt havde fortjent ei alene dette , men ogfta at

<h4>5704</h4>Saadant . Finder Han Hjertet oprigtigt og nden Falsthed i Omvendelsen , i Troen , i <b>Kjerligheden</b> , i Lengselen , i Gudftygtighedsovelserne , fta ' er Alt , hvad I gjore , Ham , velbehageligt , vm I fta blot toge et Halmstråa op as Jorden til Hans Wre , eller oplostede Eders <b>Oine</b> til Himmelen eller lagde to Skjerve i Kirkeblokken eller rakte et elendigt Memeste en Drik - koldt Vand . Dette sees blandt Andet af disse vor Frelsers Ord : <b>Oiet</b> er Legemets Lys ; derfor , dersom dit <b>Oie</b> er reent , bliver dit ganske Legeme lyst ; men dersom dit <b>Oie</b> er ondt , bliver dit ganske Legeme <b>mor</b> kt ( Matth . 6 , 22. 23 ) . Men ved <b>Oiet</b> forstaaer Han Menneskets <b>Hjerte</b> , Forstand , Villie og Hmsigter ; nåar disse ved Gnds Aand og Naade ere blcvlte velstittete , fornyede og helligede i Trom , fta ere alle dets Gjeminger og Foretagender gode og Gud velbehagelige ; er derimod Hjertet ondt og fuldt af Hykleri , fta er Alt morkt og forkastelig ! . Hertil sigter Han ogfta , nåar Han siger til Fariseerne : I ere de , som retferdiggjor e Eder <b>selv</b> for Menneskene ( I anstille Eder gudfrygtige og andegtige og paatage Eder stor Hellighed ) ; men Gnd kjender ( og bedommer ) Eders Hjerter ( Luk . 16 , 15 ) . § 49. Dette lerer Han ogfta i Lignelsen om Fariseeren og Tolderm . Hiin var ndvortes hellig og glidftygtig , og man stulde have troet , at han var eu af Guds kjereste Born ; denne derimod var efter sin udvortes Tilstand ugndelig , betynget med megm Uretferdighed og Synd ; men Gnd , som prover Hjerter og Nyrer , dommer dem Begge ester deres Hjerten Den Forsies <b>Hjerte</b> var fuldt af Hykleri , Übodferdighed , Vantro , Sikkerhed og Hofferdighet » ; derfor var hau tned al sin Skinhcllighcd en Vederstyggelighed for Gud ; deu Andens derimod var fnldt af Anger og Sorg over sine Synder , fnldt af Lengsel ester Gilds Naade og Barmhjertighed , fuldt af Udmyghcd og gode Forsetter ; derfor var han Gud tekkelig og velbehagelig . Efterdi Inu vide Saadaut, mine Kjereste , fta begynder idag at tmgte af Eders ganske Sjel og af al Eders Fvrmue efter Hjertets Oprigtighed ! Naar I gaae i Kirke , nåar I ville lede , skrifte eller forelage Eder nogen anden Gudftygtighedsovelse , fta glemmer itte at opsoge og medtage Eders <b>Hjerte</b> ! Lader dette hver Morgen og i alle Ting vere det forste Offer , I bringe Gnd ;

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016101129001'target=_>Arndt, Johann, 1838, Sex Bøger om den sande Christendom</a><h4>1396</h4>du først hvad Christus er . Bil du kjende en god Frugt og Urt , da smag den , saa veed du det ; ligesaa maa du og smage Frugten af Livets Trce Christus , om du ret stal kjende ham . Smager du i Troen hans Ydmyghed , Sagtmodighed og Taalmodighed , saa ceder du af hans Frugter , saa finder du Hvile for din Sjæl , og bliver deelagtig i hans guddommelige Trost og Naade . Men gjor du det ikke , saa finder du ingen Hvile for din Sjæl . Thi Guds Naade og Trost kan ikke stinne ind i et trolost <b>Hjerte</b> , hvori Christi Sagtmodighed og Ydmyghed ikke er ; thi han giver ikkun de Ydmyge sin Naade , I Petr . 3,5 . Hvad gavner Christus detMenneste , som intet Samfund vil have med ham ? Nu have jo de , som leve i Syndens Morke , intet Samfund med Lyset , som er Christus , derfor er han dem intet nyttig ; thi saa siger Apostelen Johannes i bemeldte Sprog : Dersom vi sige , at vi have Samfund med ham , og vandre i Msrket , da lyve vi , og gjore ikke <b>Sandhed</b> . Det forklarer han ydermere i efterfølgende Capitel , sigende : <b>Mor</b> ker er forganget , og det sande Lys skinner nu ; hvo som siger : han er i Lyset , og hader sin Broder , han er endnu i Msrket ; hvo som elsker sin Broder , han bliver i Lyset , og der er ingen Forargelse i ham . Men hvo som hader sin Broder , er i Msrket , og omgaaes i Msrket , og veed ikke hvor han gaaer ; thi Msrket har forblindet hans <b>Oine</b> . Saa længe da et Menneske bliver i saadanne Synder , saasom i et førskrækkeligt Morke , da kan det ikke blive oplyst af Christo , det sande Lys , og komme til Guds rette Kundskab . Nåar man vil ret kjende Gud og Christum , da maa man vide , at Gud og Christus er idel Naade og <b>Kjærlighed</b> . Men Ingen kan vide , hvad <b>Kjærlighed</b> er , uden den , som <b>selv</b> har <b>Kjærlighed</b> , og over sig derudi . Saa kommer og den fuldkomne og rette Kundskab om de Ting , som stulle gjores , af Forfarenhed, og at man sligt har førsogt ; derfor kommer og Kjærligheds rette og fuldkomne Kundskab af Forfarenhed . Hvo som derfor ikke over sig i <b>Kjærlighed</b> , veed ikke hvad <b>Kjærlighed</b> er , om han endstjsndt taler meget derom . Christus er idel <b>Kjærlighed</b> , Ydmyghed , Sagtmodighed, Taalmodighed , idel Dyd og Fromhed . Hvo som ikke sver sig i dem , veed ikke hvad Christus er , og kjender ham ikke ret , i hvor meget han end taler om ham , og ncevner sig efter ham . § . 3. Guds Ord er idel Aand , hvo som derfor ikke lever og vandrer i Aanden , veed ikke hvad Guds Ord er , om han end noksaa meget taler

<h4>1979</h4>§ . 2. O kjære Herre Christe , hvor mange Mennesker ere dog endnu ikke paa din <b>Sandheds</b> Vei ? " Loer mig , Herre ! din Vei , jeg vil vandre i din <b>Sandhed</b> . Giv mig et <b>Hjerte</b> , som frygter dig , en Troe , som favner dig , en <b>Kjoerlighed</b> , som efterfølger dig , et Haab , som seer til din Herlighed , et Sind , som elffer dig , en Forstand , som kjender dig , et Ore , som horer dig raabende og skrigende paa Korset i din Lidelse , et <b>Oie</b> , som seer dig i din Ydmyghed , og en Mund , som med dig beder for mine Fiender . Dersom du , Herre ! ikke laerer mig denne Vei , forer og leder mig derpaa , saa er min Vei idel Vildfarelse , og mit Lys er Morkhed . Odu evige Lys , som oplyste demgamle Tobias , der hans Syn var borte , og Isaak , der hans Sine vare morke og dumme , du som oplyste Jacob , der han forkyndte sine Ssnner tilkommende Ting , og den gamle Simeon , der han saae sin Frelser , oplys ogsaa os , at vi i Christo kunne see hans deilige Dyder stinne , og ved samme Dyder blive oplyste og beprydede . I Verdens Skabning var der morkt over Afgrunden ; ligesaa er der og morkt i vore Hjerters Grund , indtil Gud siger : vorde Lys , og indtil Guds Aand svoever over vore Hjerters Vand . Jorden var ode og tom ; ligesaa er vort <b>Hjerte</b> , dersom Guds Naade ikke opfylder det . O ! at Ordet , ved hvilket Lys og Morke bleve adskilte , vilde og i vore Hjerter , som i en ny Verden og et nyt Kreatur , fordrive Morket , og sige : vord Lys . Thi dette Ord er Lyset <b>selv</b> , og er i Christo , og er Christus <b>selv</b> , og det Lys er <b>Sandheden</b> , og uden dette Lys er alt det , som er i Menneskene , Logn og Djævelens Rige . Uden Christi Lys er der idel Morkhed udi et Menneske . Uden Christi Vei er der idel Vildfarelse i et Menneste ; uden Christi dydefulde Levnet er der idel Laster iet Menneste og den evige Død . Dette er nu Veien , dette er <b>Sandheden</b> , dette er Livet , dette er Herrens Frygt , og

<h4>2164</h4>Ringhed . Naar En seer sin egen Nsgenhed , hvor ganfle blot og tom han er fra alt Godt , da begynder han desinderligere at bede om Guds Naade og Barmhjertighed , og ret at kjende , love og prise Gud , som den rette Kilde til alt Godt ; han seer i saadan Ydmyghed Guds Herlighed og overmaade store <b>Kjærlighed</b> og Naade . Da begynde Madens Stromme at indflyde ved Bønnen i saadan en troende og ydmyg Sjæl . Ved saadan Guds Naade bliver og den Hellig-Aand jo mere og mere udgydet over os , og hans Gaver formerede , og " ved den Hellig-Aand udosts <b>Kjærlighed</b> i vore Hjerter , Rom . 5 , 5. Naar en troende Sjæl seer sin egen Ringhed og Forfængelighed , og at Guds Son dog alligevel har nedsteget saa dybt , og ikke alene er bleven et Menneske , men har endogsaa , for saadant usselt Kreaturs Skyld , som Mennesket er paa Jorden , lidt saa usigeligt , tungt , haardt og besværligt Kors . Saa bliver denne cedle Guds Kjærligheds Lue desmere brcrndende , og ved Troen opstiger tii Gud , saa at den ogsaa i Gud og Christo elster alle Mennesker for Guds store Kjærligheds Skyld ; thi den seer hvor hoilig den <b>selv</b> og alle Mennesker elskes i Christo . Og efterdi den er oploftet i Gud , og indstuttet i Guds <b>Kjærlighed</b> , saa elsker den og alt hvad Gud elsker . § . 3. Derpaa folger , at <b>Kjærlighed</b> glæder sig over alt Godt , som vor Næste vederfares , og sorger for alt Ondt ham <b>moder</b> . Og efterdi den er venlig og mild imod sin Næste , da fordommer den ikke strar sin Næste , som førsecr sig af Skrobelighed , eller derfor ilde paataler ham , meget mindre opblcestr den sig imod ham , til at foragte og forhaane ham derfor ; thi den seer i en <b>sand</b> Ydmyghed sin egen Elendighed og Blusel , og at den <b>selv</b> sidder nedscenkt i lige eller fast storre Ulykke og Synder ; og , om den staaer , da kjender den vel , at den ikke af sig <b>selv</b> , eller ved sig <b>selv</b> har kunder blive bestaaende , men at den er ved Guds Naade opholdet . Den ydmyge <b>Kjærlighed</b> dommer og straffer altid sig <b>selv</b> , for den dommer og straffer Andre , og naar den seer sin Næstes Ulykke , gaaer den ind i sit eget <b>Hjerte</b> og bejamrer sig <b>selv</b> . Thi den seer i sin Næstes Fald sine egne Feil , Synder , Jammer og Elendigheder . Bed Ydmyghed bliver et Menneske styrket i Guds Kundskab og Haabet . Thi naar et Menneske beskuer sin egen Vanvittighed , Blindhed og Daarlighed i Guds Sager , da takker det Gud for sit Ords Aabenbarelse, og tager det desmere til <b>Hjerte</b> , og naar det betragter , at al dets Formue og Kraft er intet , da styrkes det i Haabet , saa det sætter det paa Gud alene . Ved Ydmyghed bliver ogsaa et Menneske yndeligt og

<h4>2897</h4>haanden . tz . 2. Det , som Guds Born i saadan Tid og paa saadant Sted opholde sig med , er først deres Troe og Haab , som seer paa Guds Almagt og Barmhjertighed , tz . 3. De vide og , at Gud horer deres Bsnner . tz . H . At han vist troster , esterar han har bedrovet , tz . i 5. At han stal lade sit Lys stinne for dem , hvori de kunne see , at stige Ting komme fra ham <b>selv</b> ved Mennesker . § . 6. Det er og deres Trost , at <b>Sandhed</b> engang kommer for Lyset , og Falskhed staaer sin egen Herre paa Halsen . Mich . 7 , 7. : c . Jeg , jeg vil see hen til Herren ; jeg vil bie efter min Saligheds Gud ; min Gud skal hore mig . Glæd dig ikke over mig , min Modstanderinde ; naar jeg er falden , skal jeg staae op igjen ; naar jeg sidder / i <b>Mor</b> < ket , skal Herren være mit Lys . Jeg vil boere Herrens Vrede ; thi jeg har syndet imod ham , indtil han udforer min Sag , og skaffer mig Ret . Han skal udfore mig til Lyset ; jeg skal see paa hans Retfcerdighed . Dg min Modstanderinde skal see det , og Skam skal skjule hende , som siger til mig : hvor er Herren din Gud ? § . 1. Vi lcese Jer . 9 , 4. : c . at for det babyloniske Fcengsels Tid , forend Jerusalem blev første Gang førstyrret , og Israels Land odelagt, og loderne bortforte til Babylon i Fcengsel og Trceldom , have der mange vederstyggelige Laster gaact i fuld Svang iblandt loderne , saasom : Afguderie , Utroskab , Logn , Falskhed , Fiendffab , Had og Avind , og al <b>Kjærlighed</b> har været udslukt iblandt dem . Hvor det saaledes gaaer til , der er Gud ganske borte af et Land og Stad , ja ganste borte af Menneskenes Hjerter . Derpaa folger da den visse Undergang. Derfor siger Propheten Jeremias Cap . 9 , 4. 5. : " Forvarer Eder Enhver for sin Ven ; forlader Eder ikke paa nogen Broder . Thi hver Broder skuffer visselig , og hver Ven vandrer som en Bagvasker . De handle bedragclig hver med sin Ven , og tale ikke <b>Sandhed</b> . De lære deres Tunge at tale Logn ; de gjore sig megen Moie for at forvende Retten . Deres Tunge er en Slagtepiil , de talede Bedragerie ; med sin Mund taler hver vel Fred med sin Ven , men i sit <b>Hjerte</b> ligger han paa Luur efter ham , " Derved vises os i samme Ord , at der paa de Tider har været saadan Falskhed og Uoprigtighed i Landet , at intet Menneske har kunnet troe det andet , og hver Mand har beflittet sig derpaa , at han kunde besncere , bedrage og forurette sin Næste ; og at de have luret derpaa i deres Hjerter , det samme har været Aarsag til deres Fordærvelse , Ulykke og Undergang . Og derover klager og Propheten Michas Cap . 7,1 .2 . og siger : " Vee mig ! thi jeg er som de Ting , der sankes efter om Sommeren , og som efterladte Viinqviste i Viinhssten ; der er ikke een Viindrue at cede , eller den første Frugt ,

<h4>3382</h4>612 Hvorledes en troende Sjoel 3 Bogs Er nu Guds Rige ja Gud <b>selv</b> udi os , saa er han der med al sinNaade og Godhed ; thi hvor Gud og hans Rige er , der er han ganste , og did henfører han al sin Naades og Godheds Rigdom , og da er og Gud Sjoelen nærmere og mere inderlig , end den er sig <b>selv</b> . Denne naadige Guds Noerværelse , Iboelse og Rige finder og faaer man udi sig , idet man , som udi det andet Capitel er viist , soger Gud i en <b>sand</b> Omvendelse, og af ganste <b>Hjerte</b> flittig og alvorlig betienker og betragter Guds Ord , idelig og inderlig raaber til Gud om hans Naade , Oplysning og Regjering , doemver sine egne ondeAffecter , Selvkjoerlighed og Verdens-Kjirrlighed , afsiger sig <b>selv</b> ganste og holder sig übesmittet fra Verden , lader sig af Guds Villie i alting regjere , er med den i alting fornoiet og vel tilfreds , enten Gud vil have ham fattig eller rig , glad eller bedrovet , holt eller ringe begavet . Thi derved renses Hjertet fra Kreaturene og fra alle syndige Affecters Regimente og Herredomme. Og saa sinder man Gud med sit Lys og Boesen udi Sjoelcns inderste Grund . Og de , som saaledes finde Gud , hoenge ikke ved Naturen, som de blotte naturlige og uigjenfodte Mennesker , men de hoenge for , alting ved Gud , som har gjort Naturen . Og derfor ere de ogsaa over Naturen i Gud , og ere med Gud forenede , ja eet med ham . § . 2. Hvorfor de ogsaa ere de alleryndeligste Mennesker , de ere aldeles herliggjorte indvortes , som der staaer Ps . 45 , 14. , ja Guds Venneste og Deiligste , som de kaldes i Salomons Hoisang . Hvo som kunde see saadan en Sjoel , han skulde see det allerhelligste og alleryndeligste Kreatur , og moerke det guddommelige Lys at lyse og skinne i den ; thi den er med Gud forenet og er guddommelig , vel ikke efter Naturen , men dog efter Naaden . Og saadan en Sjoel begjoerer intet , enten i Tiden eller i Evigheden , uden Gud alene , ja den begjoerer Gud blot alene for alting , og begjoerer intet hsiere og mere enten i Naaden eller i Naturen . Tvertimod , om En kunde see den Sjoel , som med al sin <b>Kjærlighed</b> hoenger ved Kreaturene , der har sat sit <b>Hjerte</b> til Kjsdslyst , Oienslyst og et hossoerdigt Levnet , og har med sin <b>Kjoerlighed</b> annammet Kreaturenes Form og Skikkelse i sig , at de derudi regjere og forene sig med den , han skulde see det allergrueligste og vederstyggeligste Misfoster og Udyr for alle Hellige , han skulde moerke en ret Djoevels og Helvedes Afskyelighed og Vederstyggelighed . Og saadan sin forborgne Vederstyggelighed stal stig en afskyelig Sjoel ret see efter Dsden, naar ethvert Menneskes <b>Hjerte</b> og Samvittighed da ret stal blive blottet og aabenbaret , og Sjoelens indvortes <b>Oie</b> ret oplades ; og stal

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009082403002'target=_>Arndt, Johann, 1844, Sex Bøger om den sande Christendom</a><h4>790</h4>Kjcrrligc og yndige Glid ! du er <b>Kjcrrlighed</b> , cg den , som bliver i <b>Kjcrrlighed</b> , bliver i dig og du i ham ; ak , hvi er jeg saa vidt forfalden fra dig , at jeg . er bleven ganstc tom fra den rene brcrndendc Kjcrrlighcd , som jeg stulde berre til dig ! mit <b>Hjerte</b> er opfyldt med Kjcrrlighcd til Verden og jordiske Ting , men derimod koldt og uskikket til at elske dig , du heieste og evige Gode ! forbarm dig over mig , o barmhjertige Fader! oploft mit Sind fra Jorden , at jeg ncrrmcre og grundigere maa kjende , hvem d « er , og ret betragte dine Gjerninger mod mig , hvor heit d » t har elsket mig i Christo , og hvad for en herlig Arv du har af Naade beredt mig i ham . Lad denne Vctragtning optcende i mig en <b>sand</b> <b>Kjcrrlighed</b> til dig , som ei tilladcr mig , at hvile i nogen anden Ting , end i dig alene ; som intet agter , uden det , som kommer fra dig , og som forbinder mig fastere med dig ; som udrydder i mig al syndig <b>Kjcrrlighed</b> , ja som ov » vcrkker et hjevteligt Had mod den samme ; som og gjer mig bestandig at hcrnge ved dig i Sorg og Glcrde , at folge dig , og hverken ved Lyst eller Frygt at blive stilt ved dig . Lad og din <b>Kjcrrlighed</b> antcrndc i mig en oprigtig <b>Kjcrrlighed</b> til min Ncrste , at jeg maa , som din Gfterfolger , vandre og omgaacs i <b>Kjcrrlighed</b> , ligesom Christus har elsket os , saaledes i al min Gjoren og Laden handle med min Ncrste , som jeg vilde , at der skulde vederfares mig <b>selv</b> ; villig uden Knur hjcrlpe hau : tilrette i al Nod , berre hans Vyrde , naar han <b>selv</b> er strobelig , og hjcrlpe ham tilrette med en sagtmodig Aand , naar han forsecr sig , saa jeg maa verre den Blindes <b>Die</b> , den Haltes Fcd og de Fciderloses Fader , og det altsammen af hjertelig Kjcrrlighcd til dig , som har elsket mig i Christo Jesu , forend Verdens Gruudvold blev lagt , og jeg cndnn var din Fiende . Hor mig , min Gud ! og giv mig denne rene <b>Kjcerligheds</b> Gave , ved lesum Christmn din Eon , vor Herre , Amen .

<h4>1117</h4>herovenfra oplyste og sik den Helligaand . Men det vilde den rige Karl ( 3 uc . 18 , 23. ) ikke gjore , derfor forblev han i denne Verdens Morke , og blev ikke oplyst til det evige Liv ; thi hvo som elffer Verden , i ham er ikke Faderens <b>Kjcerlighed</b> . Derfor siger Johannes , 1 Joh . 2 , II . : " Hvo som hader sin Broder , vandrer i Morket , og er i Merket , og veed ikke , hvor han gaacr ; thi Msrket har forblindet hans <b>Oine</b> . " Derpaa driver den Kirkelcerer Taulerus i alle sine Prcrdikener : at uden en <b>sand</b> Troens Vvelse , og uden at man afdscr , forsager og fornegter sig <b>selv</b> , uden man omvender sig af sit ganske <b>Hjerte</b> , og giver sig ganske Gud i Vold , kan man ingenlunde finde i sig det guddommelige Lys .

<h4>1631</h4>S ^ ndheds Vei ? Lcrr mig , Herre ! din Vei , jeg vil vandre i din <b>Sandhed</b> . Giv mig et <b>Hjerte</b> , som frygter dig , en Troe , som favner dig , en <b>Kjcerlighed</b> , som eftcrfslger dig , et Haab , som seer til din Herlighed , et Sind , som elsker dig , en Forstand, som kjender dig , et Ore , som horer dig raabende og strigende paa Korset i din Lidelse , et Vie , som seer dig i din Vdmyghed , og en Mund , som med dig beder for mine Fiender . Dersom du , Herre ! ikke locrer mig denne Vei , forer og leder mig derpaa , saa er min Vei idel Vildfarelse , og mit Lys er Msrkhed . Odu evige Lys , som oplyste den gamle Tobias , der hans Syn var borte , og Isaak , der hans <b>Oine</b> vare msrke og dumme , du som oplyste Jacob , der han forkyndte sine Sonner tilkommende Ting , og den gamle Simeon , der han saae sin Frelser , oplys ogsaa os , at vi i Christo kunne see hans deilige Dyder stinne , og ved samme Dyder blive oplyste og beprydede . I Verdens Skabning var der morkt over Afgrunden ; ligesaa er der og msrkt i vore Hjerters Grund , indtil Gud siger : vord Lys , og indtil Guds Aand svcever over vore Hjerters Vand . Jorden var ode og tom ; ligesaa er vort <b>Hjerte</b> , dersom Guds Naade ikke opfylder det . O ! at Ordet , ved hvilket Lys og Morke bleve adskilte , vilde og i vore Hjerter , som i en ny Verden og et nyt Kreatur , fordrive Morket , og sige : v o r d 3 y s . Thi dette Ord er Lyset <b>selv</b> , og er i Christo , og er Christus <b>selv</b> , og det Lys er <b>Sandheden</b> , og uden dette Lys er alt det , som er i Menneskene , Logn og Djcevelens Rige . Uden Christi Lys er der idel Morkhed udi et Menneske . Uden Christi Vei er der idel Vildfarelse i et Menneffe ; uden ChriKi dydefulde Levnet er der idel Laster i et Menneske og den evige Dod . Dette er nu Veien , dette er <b>Sandheden</b> , dette er Livet , dette er Herrens Frygt , og d e t E n e , som David onsker sig , at have Gud og Christum stedse i Hjertet ved Troen , at eftcrfolge Christum i <b>Kjoerlighed</b> , Ydwyghed og Sagtmodighed , at have Christum stedse for Sine , som en Vei og Speil at leve efter , og i Haabet stedse beskue hans Herlighed ; det er den Herrens Frygt , som bevarer Menneskene , som gjor Hjertet fromt , og afvender Synden ; thi hvo som lever uden Frygt , han kan ikke behage Gud , og hans egen Dristighed stal nedstyrte ham . Uden Herrens Frygt duer hverken Konst eller Rigdom , Deilighet ) eller Styrke , Klogstab eller Veltalenhed ; thi Naturens Gaver have de Onde og Gode tilfoelles , og de sindes i dem begge ; men Herrens Frygt er en scerdeles Gave , et synderligt Lys , hvilket gjor os til Guds Venner, ja til Guds Born , som behage Gud formedelst Troen . Gud har ingen Behagelighed til Klogstab , Konst , Forstand , Veltalenhed , Rigdom og Dcilighed , naar Gudsftygt ikke er derhos . Rigdom uden <b>Kjcerlighcd</b> , Konst uden Naade og Deilighed uden Gudsftygt ere ligesom en Skygge uden Legemet , som Avner uden Hvede , Ax uden Korn , og som Rog uden Ild . Gudsftygt er alle Gavers Prydelse . Den er de Ringes Krone , de Vises Raadgiver , de Deiliges Held , de Stcerkes Seier, Fyrsternes Mre , Vorns Tugtem ester . Den bevarede og frelste Daniel , Joseph og Susanna . Uden Gudsftygt er et Menneske et tort Troe , en unyttig Brand , et ssnderstaget Skaar , som man udkaster . " Store Herrer , Dommere og moegtige Mcend stulle « res , men der er Ingen af dem sterre , end den , der frygter Herren . " Sir . 10 , 27.

<h4>3264</h4>Herre min Gud , fra hvilken , som fra en Lysenes Fader , alle gode Gaver komme heroven ned , send du dit Lys og din <b>Sandhed</b> i min Sjcrl , at de kunne lede og fere mig fra al Krcatur-Kjcrrlighcd til dig <b>selv</b> og til din og min Ncrstes rene <b>Kjcrrlighed</b> . Virk du kraftig i mig , at alle mine Gjerninger maatte blive gjorte i dig , komme fra dig , som fra den ferste Kilde og Vegyndelse , og sigte alene til din Mre . Du er <b>Kjcrrlighed</b> , gjor mig dig herudi liig , at jeg og maa eve en ret Kjcrrlighcd imod min Ncrste . Min fordcrrvede Vanartighed forleder mig ofte til at fcrlde storagtige og foragtelige Domme om min Ncrste , derfor nnddragcr du mig og ofte din Naade , paa det jeg kan lcrre , i hvilken Kraft jeg staacr , og at jeg ikke har at rose mig af det , som jeg har faaet , ligesom jeg ikke havde annammet det af Nogen . Tilgiv mig , min Gnd , og hjcrlp mig fra Synden . Giv mig din Aand , der kan lcrre mig at holde ret Maade , at jeg hverken paa den ene Side kalder det Onde Godt , gjor Morke til Lys , og derved drager Vec over mig , ei ^ heller at jeg paa den anden Side opheier og hovmoder mig af en phariscrisk Aand over min Ncrstc , som er falden , har et Velbehag til mig <b>selv</b> , og foragtcr Andre i Henseende til mig . Lcrr mig at bctcenke , o Herre , at det er din Naade at tilskrive , naar jeg staaer , men at jeg og vel kan falde , og visselig maatte falde , dersom du tog din Haand tilbage . Lad mig betragte , hvor meget der endnu fattes mig <b>sclv</b> , at der derved maa opvcrkkes en Medlidenhed udi mig imod min svage Ncrste . Saa ofte som min fordcrrvede Vanart viser mig Skjerven i min Ncrstes 33 ie , saa viis du mig af din Naade den Vjcrlke , som er i mit eget drag du den tilforn nd af mit <b>Die</b> , og indskriv tillige udi mit <b>Hjerte</b> de Torden-Ord : O Menneske , du kan ikke nndskylde dig , hvem du og er , du som dommer en Anden ; thi hvorudi du demmer Andre , deri fordommer " du dig <b>selv</b> , eftcrdi du gjer de selvsamme Ting , som du dommer Andre for ; men tcrnkcr du dette , o Menncffe , du som demmer dem , der gjere saadannc Ting , og gjor <b>selv</b> de samme , at du skal undflye Gnbs Dom ? Men dersom min Ncrstes ringe Forseelse endelig ferer det med fig , at den maa blive demt , saa giv , at jeg maa demme en Kjcrrligheds-Dom , der maa udfalde til hans og andro Menneskers Forbedring , der knnde maaskee have forarget fig over ham ; lad ung bede for ham , og saaledes ikke sege mig <b>selv</b> eller min Anseelse , men din Mre og Ncrstens Velfcrrd , ved lesum Christum , din Sen , vor Herre , som har herudi efterladt os et Exempcl , Amen .

<h4>3414</h4>Ak , du min hjcrte-clffcnde Fader i Himmelen , hvilken en stor <b>Kjcrrlighed</b> har du beviist mig deri , at jeg skulde verre dit Varn , og du vilde verre min Fader , at du fot din kjerre Sons vor Herres Jesu Christi Skyld har antaget mig at verre et Varn og Arving til alle dine himmelske gode Ting ; giv mig et sonligt <b>Hjerte</b> imod dig , ligesom du har <b>selv</b> et trofast Fader-Hjcrtc imod mig , saa jeg fatter al min Tillid paa dig , hjertelig elsker dig , sonlig frygter dig , ydmygclig crrer dig , og er dig lydig i alle Ting , at jeg har min hoieste Lyst , Fryd og Glcrde i dig . Og naar jeg er fattig og elendig , naar jeg er syg og fonolges , naar jeg geraader i Dods-Nod , saa lad mig Elendige tcrnke paa dig , at jeg har en Fader , som er almcrgtig , barmhjertig , naadig , taalmodig , og af stor Godhed , der ikte stal forglemme mig , ligcsaalidt som en <b>Moder</b> kan forglemme sit diende Varn ; og naar jeg synder af Skrobelighed , da ikke forsage , men vende om og sige : Fader ! jeg har syndet mod Himmelen og for dig . Hjcrlp og , at alle Mennesker paa Jorden med mig maa erkjende dig for en Fader , paakalde dig med rene Hjerter , og love dig som med cen Mund , saa vi samle alle vore Bonner sammen , og Alle bede for hverandre . Ak Fader ! efterdi du er i Himmelen , saa lad mig have min Vandel i Himmelen , paa det nut <b>Hjerte</b> ikke maa hcrngc ved det Timelige, men at jeg maa erkjende , at jeg er en Pilegrim og Fremmed paa Jorden , ligesom alle mine Fcrfcrdre . Sce , du er jo hos mig , ja allesteds ncrrvcrrendc ; du er ikke en Gud , som er langt fra mig , men som er ncrr hos . Du er en Gud , som opfylder Himmelen og Jorden , derfor kan man paakalde dig til alle Tider og alle Steder . Ak Gud , vi dine arme Born ere paa Jorden i denne Jammerdal , men du er i Himmelen, i dine Helliges herlige Bolig , vi sukke til dig , og haabc at komme til dig , vor Fader . Ak , hjcrte-elskende Fader , cfterdi dn er en hellig Gud , og dit Navn er stort og mcegtigt , saa giv mig , at jeg og alle Mennesker maa af dit Ord kjende dig ret , du Gud Fader , Son og Hcllig-Acmd , prise dit store Navn , din Almcrgtighed , Barmhjcrtighcd, Godhcd , Netfcrrdighed , Viisdom og <b>Sandhed</b> , alle Tider crre og prise dig i mit <b>Hjerte</b> med alle Troende , saa at din Lov aldrig mere maa komme af mit <b>Hjerte</b> og Mund , at min Forstand maa oplyses ved dig , at min Villie maa elske dig over Alting , at jeg uden Afladclsc tcrnker med hjertelig Taksigclse og Fryd paa din Kjcrrlighcd og Troffab , som du har bcviist mod mig , dit elendige Varn . Giv mig en utrettelig Flid til at soge dig , giv mig Viisdom til at finde dig , giv mig et Levnet , som behager dig , paa det at al Mre i alle Ting maa gives dig ved mig og alle Mennesker. Ak , kjerre Fader ! cfterdi dit Nige er det hoiestc Gode og ' den hoicste Skat , saa lad det tromme til mig og alle Mennesker ved den Hcllig-Aand , at du ved Troen kan boe i mig , have dit Nige i nn ' g > at jeg med en hjertelig <b>Kjcrrlighed</b> kan omfavne dig , med et levende Haab hcrngc ved dig , paa det jeg maa blive dcclagtig i alle dit evige Nigcs gode Ting , din Rctfcrrdighed , din Fred , din Fryd i den Hellig-Aand , paa det at Satans Rige , Synden og alle Djcrvclcns Gjerninger derved i mig og alle Mennesker kan tilintctgjores , som Hoffcrrdighed , Gjerrighed , Vrede og Kjodcts urene Lyster , hvori Satan har sit Nige . Ak , kjerre Fader ! cfterdi Intet er hclligcre end dit Navn , Intet kosteligerc end dit Rige og Intet bedre end din Villie , saa giv mig , at din Villic maa skee i mit og alle Menneskers Hjerter , at jeg af ganske <b>Hjerte</b> begjcrrcr dig , soger og erkjender dig , og gjor Alt det , som dig behager . Derfor indret og forordne Alt mit Vcrscn og Levnet , min Gjoren og Laden , til dit Navns Lov og Priis . Giv mig , at jeg maa vide , ville og kunne gjore " alle Ting , som dig velbehagcr , og som tjener til din Mrc og min Salighcd . O hjcrte-elffendc Fader ! viis mig en ret , jcrvn og sikker Vei til dig , og det verre dig ovcrladj , hvordan du vil fore mig , igjenncm Velstand eller Vecstand , at jeg i de gode Dage maa love og takke dig , og ei ophoic mig , men dcrimod verre taalmodig i min Vidcrvcrrdighcd og ikke forsage . Forlecn mig dette , at intet maa glcrde mig , uden det , som forer mig til dig , ei heller noget bedrove mig , uden det , som afvendcr mig fra dig . Giv mig , min Gud , at jeg ikke maa begjcrre at takkes Nogen , uden din Villic , og at det af ganske <b>Hjerte</b> maa behage mig , som er dig kjccrt og angcncmt , og at jeg maa have et Mishag til al timelig Glcede , som er uden dig , og at jeg intet begjcrrer , som er dig imod . Giv mig en saadan Lyst til dig , saa du maa verre mig alle Ting , at jeg intet onsker og begjcrrcr , uden dig , paa det jeg mcd Glcrde kunde sige : Herre , naar jeg kun har dig , saa ffjottcr jeg hverken om Hunmcl 31 *

<h4>3484</h4>beredt til al god Gjerning , " 2 Tim . 3 , 17. Og dersom det er en Gudstjeneste, som du vil gjsre , da stal du vaere vis paa , at Gud i sit Ord har befalet den Sag , som du vil gjsre ; thi " de dyrke mig forgjeves , idet de lcere saadanneLcerdomme, som ere Menneskenes Bud , " siger Christus Matth . 15 , 9. Og er det en anden gemeen Gjerning , da stal du i det ringeste af Guds Ord verre vis paa , at den er tilladt og lovlig ; thi tvivlcr man paa Noget , om det er lovligt , og gjor det dog , saa bliver det til idel Synd . " Enhver verre fuld forvisset i sit eget Sind og Mening," siger Paulus Norn . 14 , 5. Item 23 V . : " Hvo som tvivler paa Noget , og gjor det dog , han er fordomt , thi det er ikke af Troen , og Alt det , som ikke er af Troen , er Synd . " Derncest stulle alle dine Gjerninger stee ien <b>sand</b> Gudsfrygt og <b>Kjoerlighed</b> ; thi deri er og Troen befattet . Og " uden Troen er det umuligt at velbehage Gud , " Ebr . 11 , 6. Derfor siger og GudsAand til de Ebrceer Cap . 11 , 4. : " At Abel offrede Gud formedelst Troen et stsrre Slagtoffer end Cain . " Og i Ordsprogene Cap . 15 , 8. : " At de Ugudeliges Offer er Herren en Vederstyggelighed." For det tredie , stal du gjore Alt , hvad du gjer til Guds Mre . " Gjor Alting til Guds Mre , " siger Paulus 1 Cor . 10 , 13. Guds Mre stal for Alting ssges , og ikke vor egen Mre . Hvad et Menneske gjor til sin egen Mre , og soger derved fornemmelig , at det af Menneskene derfor kan « res , ansees og hoit agtes , det kan ingenlunde tcekkes og behage Gud , i hvor herligt det end udvortes kan synes; thi det kommer ikke af Gud , men af Menneskenes egen Hovmodighed og Opblcrselse, og tilhsrer derfor ikkun Ens Mrgjerrighed og Selvbehag . Thi , hvo som har gjort og afstedkommet Gjerningen , den tilhorer Gjerningen ; og hvo som har fodt eller avlet Fosteret , den tilhorcr og Fosteret . Den som udi sine Gjerninger mener og soger sig <b>selv</b> , og soger dcrudi sin egen Mre , og ikke for Alting ssger Guds Mre , han er en Hykler og Dienstalk . Og derfor ere og stige hykkelste Gjerninger " ligesom de kalkede Grave , som synes deilige udvortes , men ere indentil fulde af dode Been og Ureenhed , " Matth . 23 , 27. De ere og ligesom Kul og Skarn , der er udentil forgyldt , men straber man Guldet af , saa bliver det andet Skarn alene tilbage ; thi scetter man det udvortes Skin tilside , som sindes udi stige Hykkelste Gjerninger , saa bliver Intet tilovers , uden Skam og Skjamdsel ; nemlig den vederstyggelige Mrgjerrighed og blinde <b>Kjcrrlighed</b> til sig <b>selv</b> , som vel for det hovmodige og vildfarende <b>Hjerte</b> synes en yndig og behagelig Ting , men er dog for Gud en Afstye og Vederstyggelighet Ia det befindes i <b>Sandhed</b> et idel Afguderie; thi Alt det man soger og anseer enten lige ved Gud , eller mere end Gud , det er altsammen Ens Afgud , og alt det man derudi gjor , er idel Afguderje . Hvorledes skulde det da behage Gud , som er en stcrrk og nidkjcer Gud , der ikke kan fordrage det Onde , og der ikke vil give nogen Anden sin Mre , eller Afguderne sin Lov og Priis , Esa . 42 , 8. Ydermere og for det fjerde , stal du og gjsre alle dine Gjerninger i <b>Kjcerlighed</b> til din Naeste . Soger du din Ncestes Skade med den Gjerning du gjor , da kan den ikke mere behage Gud , end det kunde blandt Menneskene vcere behageligt , at du stagtede Ssnnen for Faderens <b>Oine</b> . Og veed du , at en Gjerning i sig <b>selv</b> er lovlig , og du veed tillige , at din Nceste , som ikke har den Kundstab , at den er lovlig , kan forarges derved , da stal du heller lade verre saadan Gjerning , end derved forarge din Nwste ; thi man stal ikke foette Stod og Forar-

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016051048081'target=_>Arndt, Johann, 1855, Sex Bøger om den sande Christendom</a><h4>621</h4>§ 3 . Findes nu denne sande Guds Kundskab udi os , ved hvilken Gud giver sig vor Sjæl ligesom til at smage , som en Mad og Spise , at den 34 Psal . 35. 9. derfor siger : " Smager og seer , hvor sod og venlig Herren er . " saa kan det ikke feile , der jo folger en <b>sand</b> Omvendelse derpaa , det er , Sindets Forandring og Fornyelse og Livs og Levnets Bedring . Thi naar Nogen har ret * ) kjendt og funden Guds Almægtighed i sit <b>Hjerte</b> , saa folger deraf Ydmyghed , at man ydmyger og fornedrer sig under Guds vceldige Haand . Har Nogen ret smagt og kjendt Guds Barmhjertighed, da folger derpaa en brcrndende <b>Kjærlighed</b> til vor Næste . Thi Ingen , som kjender ret Guds Barmhjertighed , kan være übarmhjertig , og hvorledes skulde den kunne vaere haard og umiskundelig imod sin Næste , som finder Guds rige Naade og Barmhjertighed mod sig <b>selv</b> ? og deraf folger ogsaa Taalmodighed med vor Næste , hvilken er saa grundig og hjertelig , at om en ret Christen blev end myrdet og drcebt syv Gange om Dagen , og blev syv Gange levende igjen , da tilgav han dog for Guds store Barmhjertigheds Skyld sin Fiende al den Uret . Af Guds Retfærdigheds Kundskab flyder Syndens Kundskab , at man siger med Propheten Dan . 9 , 7 : " Herre , dig horer Retfcerdighed til , men os vort Ansigtes Blussel . " Item : " Gaae ikke i Dom med din Tjener ; thi Ingen , som lever , kan være retfcerdig for dig , " Ps . 143,2 . ; og Ps . 130 , 3. : " Herre , dersom du vil tage vare paa vore Misgserninger , Herre , hvo kan da bestaae ? " Af Guds <b>Sandheds</b> Kundskab flyder Trostab mod vor Næste , og fordrives af Falskhed , Uoprigtighed , Bedragerie og Logn , aten Christen tcenker : see , handler du uoprigtig med din Næste , da synder du mod Guds evige <b>Sandhed</b> , som er Gud ftlv . Efterdi Gud derfor handler trofast og sanddru med dig , saa maa du handle lig / saa med din Næste . Af Guds evige Viisdoms Kundskab flyder Gudsfrygt ; tyl efterdi du veed , at Gud randsager Hjerter og Nyrer , og seer alle Ting , i hvor stjulte de end kunne være , saa frygter du dig billig for hans hellige Majestcets Vine . " Mon den , som plantede Oret , stulde ikke hore ? eller den , som dannede <b>Oiet</b> , stulde han ikke see ? " P 5.94 , 4. " Vee dem , som scenke sig dybt fra Herren , at skjule deres Anslag , og hvis Gjerninger ere i Morket , og sige : hvo seer os ? og hvo kjender os ? O Eders forvendte Handel ! mon Pottemageren stal agtes som Leer , at Gjerningen kunde sige til den , som gjorde den : han har ikke gjort mig , eller at det , som er dannet , kunde sige til den , som dannede det : han forstod det ikke ? " Es . 29,15 . 16. Jer . 23 , 24. ; 32 ,

<h4>1048</h4>herovenfra oplyste og fik den Helltgaand . Men det vilde den rige Karl ( Luc . 18 , 23. ) ikke gjore , derfor forblev han i denne Vardens Morke , og blev ikke oplyst til det evige Liv ; thi hvo som elsker Verden , i ham er ikke Faderens <b>Kjærlighed</b> . Derfor siger Johannes , I Joh . 2 , 11. : " Hvo som hader sin Broder , vandrer i M orket , og er i Morket , og veed ikke , hvor han gaaer ; thi Merket bar forblindet hans <b>Oine</b> . " Derpaa driver den Kirkelærer Taulerus i alle sine Prædikener : at uden en <b>sand</b> Troens Vvelse , og uden at man afdser , forsager og fornegter sig <b>selv</b> , uden man omvender sig af sit ganske <b>Hjerte</b> , og giver sig ganske Gud i Vold , kan man ingenlunde finde i sig det guddommelige Lys .

<h4>1551</h4>Sandheds Vei ? Lcrr mig , Herre ! din Vei , jeg vil vandre i din <b>Sandhed</b> . ssiv mig et <b>Hjerte</b> , som frygter dig , en Troe , som favner dig , en <b>Kjærlighed</b> , som efterfolger dig , et Haab , som seer til din Herlighed , et Sind , som elsker dig , en For « stand , som kjender dig , et Dre , som horer dig raabende og skrigende paa Korset i din Lidelse , et Vie , som seer dig i din Vdmyghed , og en Mund , som med dig beder for mine Fiender . Dersom du , Herre ! ikke lærer mig denne Vei , forer og leder mig derpaa , saa er min Vei idel Vildfarelse , og mit Lys er Morkhed . Odu evige Lys , som oplyste den gamle Tobias , der hans Syn var borte , og Isaak , der haus Vine vare morte og dumme , du som oplyste Jacob , der han forkyndte sine Sonner tilkommende Ting , og den gamle Simeon , der han saae sin Frelser , oplys ogsaa os , at vi i Christo kunne see hans deilige Dyder skinne , og ved samme Dyder blive oplyste og beprydede . I Verdens Skabning var der msrkt over Afgrunden ; ligesaa er der og msrkt i vore Hjerters Grund , indtil Gud siger : vord Lys , og indtil Guds Aand svcrver over vore Hjerters Vand . Jorden var ode og tom ; ligesaa er vort <b>Hjerte</b> , dersom Guds Naade ikke opfylder det . O ! at Ordet , ved hvilket Lys og Morte bleve adskilte , vilde og i vore Hjerter , som i en ny Verden og ct nyt Kreatur , fordrive Morkel , og sige : vord Ly s . Thi dette Ord er Lyset <b>selv</b> , og er i Christo , og er Christus <b>selv</b> , og det Lys er <b>Sandheden</b> , og uden dette Lys er alt det , som er i Menneskene , Logn og Djævclens Rige . Uden Christi Lys er der idel Morkhed udi et Menneske . Uden Christi Vei er der idel Vildfarelse i et Menneske ; uden Christi dydefulde Levnet er der idel Laster i et Menneske og den evige Dod . Dette er nu Veien , dette er <b>Sandheden</b> , dette er Livet , dette er Herrens Frygt , og det Ene , som David onster sig , at have Gud og Christum stedse i Hjertctved Troen , at efterfolge Christum i <b>Kjærlighed</b> , Ydmyghed og Sagtmodighed , at have Christum stedse for <b>Oine</b> , som en Vei og Speil at leve efter , og i Haabet stedse beskue hans Herlighed ; det er den Herrens Frygt , som bevarer Mennestene , som gjor Hjertet fromt , og afvender Synden ; thi hvo som lever uden Frygt , dan kan ikke behage Gud , og hans egen Dristighed stal nedstyrte ham . Uden Herrens Frygt duerhverken Konst eller Rigdom , Deil ' ghed eller Styrke , Klogstab eller Veltalenhed ; thi Naturens Gaver have de Onde og Gode tilfcelles , og de findes i dem begge ; men Herrens Frygt er en scerdeles Gave , et synderligt Lys , hvilket gjor os til Guds Venner, ja til Guds Born , som behage Gud formedelst Troen . Gud har ingen Behagelighed til Klogstab , Konst , Forstand , Veltalenhed , Rigdom og Deilighed , naar Gudsfrygt ikke er derhos . Rigdom uden <b>Kjærlighed</b> , Konst uden Naade og Deilighed uden Gudsfrygt ere ligesom en Skygge uden Legemet , som Avner uden Hvede , Ar uden Korn , og som Rog uden Ild . Gudsfrygt er alle Gavers Prydelse . Den er de Ringes Krone , de Vises Raadgiver , de Deiliges Held , de Stcerkes Seier , Fyrsternes ATre , Borns Tugtemester . Den bevarede og frelste Daniel , Joseph og Susanna . Uden Gudsfrygt er et Menneske et tort Trce , en unyttig Brand , et ssnderslaget Staar , som man udkaster . " Store Herrer , Dommere og mægtige Mænd stulle ceres , men der er Ingen af dem storre , end den , der frygter Herren . " Sir . 10,27 .

<h4>3061</h4>Herre min Gud , fra hvilken , som fra en Lysenes Fader , alle gode Gaver komme heroven ned , send du dit Lys og din <b>Sandhed</b> i min Sjæl , at de kunne lede og føre mig fra al Krcatur-Kjoerlighed til dig <b>selv</b> og til din og min Næstes rene <b>Kjærlighed</b> . Virk du kraftig i mig , at alle mine Gjerninger maatte blive gjorte i dig , komme fra dig , som fra den , første Kilde og Begyndelse / og sigte alene til din 3 Ere . Du er <b>Kjærlighed</b> , gjsr mig dig herudi liig , at jcg og maa sve en ret <b>Kjoerlighed</b> imod min Noeste . Min fordcrrvede Vanartighed forleder mig ofte til at fcelde storagtige og for- , agtelige Domme om min Næste , derfor unddrager du mig og ofte din Naade , vaa det jeg kan Icere , i hvilken Kraft jeg staaer , og at jeg ikke har at rose mig af det , som jeg har faaet , ligesom jeg ikke havde annammet det af Nogen . Tilgiv mig , min Gud , og hjælp mig fra Synden . Giv mig din Aand , der kan lære mig at holde ret Maade , at hverken vaa den ene Side kalder det Onde Godt , gjsr Msrke til Lys , og derved drager Vee over mig , ei heller at jeg vaa den anden Side ophsier og hovmoder mig af en vharisceist Aand over min Næste , som er falden , har et Velbehag til mig <b>selv</b> , og foragter Andre i Henseende til mig . Loer mig at betcenke , o Herre , at det er din , Naade at tilskrive , naar jeg staaer , men at jeg og vel kan falde , og visselig maatte falde , dersom du tog din Haand tilbage . Lad mig betragte , hvor meget der endnu fattes mig <b>selv</b> , at der derved maa opvcekkes en Medlidenhed udi mig imod min svage Næste . Saa ofte som min fordcrrvede Vanart viser mig Skjæven i min Næstes <b>Oie</b> , saa viis du mig af din Naade den Bjælke , som er i mit eget <b>Oie</b> ; drag du den tilforn ud af mit ' <b>Die</b> , og indskriv tillige udi mit <b>Hjerte</b> de Torden-Ord : O Menneske , du kan ikke undskylde dig , hvem du og er , du som dommer en Anden ; thi hvorudi du dommer Andre , deri fordsmmer du dig <b>selv</b> , efterdi du gjsr de selvsamme Ting , som du dommer Andre for ; men tcenker du delte , o Menneste , du som dommer dem , der gjsre saadanne Ting , og gjsr <b>selv</b> de samme , at du stal undfiye Guds Dom ? Men dersom min Næstes ringe Forseelse endelig forer det med sig , at den maa blive dsmt , saa giv , at jeg maa dsmme en Kjærligbeds-Dom , der maa udfalde til hans og andre Menneskers Forbedring , der kunde maaskee have forarget sig over ham ; lad mig bede for ham , og saaledes ikke ssge mig <b>selv</b> eller min Anseelse , men din Mre og Næstens Velfcerd , ved lesum Christum , din Ssn , vor Herre , som har herudi efterladt os et Exempel , Amen .

<h4>3107</h4>ligger der hsi Magt paa , at du elster Gud , paa det du kan hsre ham i dit <b>Hjerte</b> tale i din Sjæl . " Men vil du vide ( siger den Kirkelærer Gregorius ) om du elster Gud , da giv Agt paa , om du og med Taalmodighed optager af Gud alt Kors og Lidelse , Trcengsel og Elendighed , og ikke knurrer imod Gud , og er utaalmodig derover, eller for stig Modgangs Skyld afviger fra Gud , og forlader hans Bud og <b>Kjærlighed</b> . Tager du dig vare for saadan Utaalmodighed , da elster du Gud ; men viger du derover fra Gud , fordi han tilskikker dig Modgang , da elster du ikke Gud , men elster kun dig <b>selv</b> , du elster mere Vellyst end Gud , og Gaverne mere end den , som gav dem . " Elster du nu Gud hjertelig , da faaer du mangt et ssdt og saligt Ord af ham i dit <b>Hjerte</b> at hore ; thi han siger jo : " Hvo mig elster , forden viljegaabenbaremigselv ; " hvilken Aabenbarelse steer derved , at Forstanden formedelst " Viisdommens og Forstandens Aand , Kundskabens og Herrens Frygtes Aand " oplades , og vor Fornufts Dine oplyses , til at see og kjende Christum . Eph . 1,18 . § 2 . Ydermere , ligesom Djævelen ved sin Indgivelse tilstopper de indvortes Oren , som tilforn er meldt , saa forblinder han og de indvortes <b>Oine</b> . Og det gjsr han ved Verdens og Kreaturenes <b>Kjærlighed</b> , og fornemmelig med Selvkjærlighed, Opblcrselse , Hoffcerdighed og Hovmodighed , hvilke ganste forhindre dig fra at see og fornemme Christi Yndighed og 2 Eoelhed . Thi ligesom du ien hjertelig og inderlig <b>Kjærlighed</b> maa hsre Christum tale i dig , saa maa du ogsaa i Troen og en <b>sand</b> Ydmyghed , hvormed Hjertet renses fra den lede Opblcrselse og Hovmodighed , see Christum i dig ; derfor siger Chrisius Matth . 5 , 8. : " Salige

<h4>3179</h4>Ak , du min hjerte-elskende Fader i Himmelen , hvilken en stor <b>Kjærlighed</b> har du beviist mig deri , at jeg skulde være dit Oarn , og du vilde være min Fader , at du for din kjære ' Ssns vor Herres Jesu Chrisii Skyld har antaget mig til at være et Barn og Arving til alle dine himmelske gode Ting ; giv mig et ssnligt <b>Hjerte</b> imod dig , ligesom du har <b>selv</b> et trofast Fader-Hjerte imod mig , saa jeg scetter al min Tillid paa dig , hjertelig elsker dig , ssnlig frygter dig , ydmygelig oerer dig , og er dig lydig i alle Ting , at jeg har min hoieste Lyst , Fryd og Gloede i dig . Og naar jeg er fattig og elendig , naar jeg er syg og forfolges , naar jeg geraader i Døds-Nød , saa lad mig Elendige tcenke paa dig , at jeg har en Fader , som er almægtig , barmhjertig , naa > ig , taalmodig , og af stor Godhed , der ikke stal forglemme mig , ligesaalidt som en <b>Moder</b> kan forglemme sit diende Barn ; og naar jeg synder afSkrsbelighed , da ikke forsage , men vende om og sige : Fader ! jeg har syndet mod Himmelen og for dig . Hjælp og , at alle Mennesker paa Jorden med mig maa erkjende dig for en Fader , paakalde dig med rene Hjerter , og love dig som med een Mund , saa vi samle alle vore Bonner sammen , og Alle bede for hverandre . Ak Fader ! efterdi du er i Himmelen , saa lad mig have min Vandel i Himmelen , paa det mit <b>Hjerte</b> ikke maa hcenge ved det Timelige, men at jeg maa erkjende , at jeg er en Pilegrim og Fremmed paa Jorden , ligesom alle mine Forfædre . Eee , du er jo hos mig , ja allesteds nærværende ; du er ikke en Gud , som er langt fra mig , men som er noer hos . Du er en Gud , som opfylder Himmelen og Jorden , derfor kan man paakalde dig til alle Tider og alle Steder . Ak Gud , vi dine arme Bsrn ere paa Jorden i denne Jammerdal , men du er i Himmelen, i dine Helliges herlige Bolig , vi sukke til dig , og haabe at komme til dig , vor Fader . Ak , hjerte-elskende Fader , efterdi du er en hellig Gud , og dit Navn er stort og moegtigt , saa giv mig , at jeg og alle Mennesker maa af dit Ord kjende dig ret , du Gud Fader , Son og Hellig-Aand , prise dit store Navn , din Almoegtighcd , Barmhjertighed, Godhed , Retfcerdighed , Viisdom og <b>Sandhed</b> , alle Tider cere og prise dig i mit <b>Hjerte</b> med alle Troende , saa at din Lov aldrig mere maa komme af mit <b>Hjerte</b> og Mund , at min Forstand maa oplyses ved dig , at min Villie maa elske dig over Alting , at jeg uden Afladelse tcenker med hjertelig Taksigelse og Fryd paa din <b>Kjærlighed</b> og Troskab , som du har beviist mod mig , dit elendige Barn . Giv mig en utrcettelig Flid til at soge dig , giv mig Viisdom til at sinde dig , giv mig et Levnet , som behager dig , paa det at al Wre i alle Ting maa gives dig ved mig og alle Mennesker. Ak , kjære Fader ! efterdi dit Nige er det hoieste Gode og den hoieste Skat , saa lad det komme til miq og alle Mennesker ved den Hellig-Aand , at du ved Troen kan boe i mig , have dit Nige i m ' g , at jeg med en hjertelig <b>Kjærlighed</b> kan omfavne dig , med et levende Haab hcenge ved dig , paa det jeg maa blive deelagtig i alle dit evige Riges gode Ting , din Nctfcerdighed , din Fred , din Fryd i den Hellig-Aand , paa det at Satans Rige , Synden og alle Djævelens Gjerninger derved i mig og alle Menncster kan tilintetgjsres , som Hoffcerdighed , Gjerrighed , Vrede og Kjodets urene Lyster , hvori Satan har sit Rige . Ak , kjoere Fader ! efterdi Intet er helligere end dit Navn , Intet kosteligere end dit Rige oq Intet bedre end din Villie , saa giv mig , at din Villie maa skce i mit og alle Menneskers Hjerter , at jeg af ganske <b>Hjerte</b> begjærer dig , soger og erkjender dig , og gjor Alt det , som dig behager . Derfor indret og forordne Alt mit Vcrsen og Levnet , min Gjsren og Laden , til dit Navns Lov og Priis . Giv mig , at jeg maa vide , ville og kunne gjore alle Ting , som dig velbehager , og som tjener til din Mre og min Salighed . O hjerte-elskende Fader ! viis mig en ret , jcrvn og sikker Vei til dig , og det være dig overladt , hvordan du vil fere mig , igjennem Velstand eller Veestlmd , at jeg i de gode Dage maa love og takke dig , og ei ophoie mig , men derimod være taalmodig i min Viderværdighed og ikke forsage . Forleen mig dette , at intet maa gloede mig , uden det , som forer mig til dig , ei heller noget bedrove mig , uden det , som afvender mig fra dig . Giv mig , min Gud , at jeg ikke maa begjære at tcekkes Nogen , uden din Villie , og at det af ganske <b>Hjerte</b> maa behage mig , som er dig kjært og angenemt , og at jeg maa have et M.shac / til al timelig Gloede , som er uden dig , og at jeg intet begjærer , som er dig imod . Giv mig en saadan Lyst til dig , saa du maa være mig alle Ting , at jeg intet onster og begjærer , uden dig , paa det jeg med Gloede kunde sige : Herre , naar jeg kun har dig , saa stjotter jeg hverken om Himmel 31 *

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016100729002'target=_>Arndt, Johann, 1844, Sex Bøger om den sande Christendom</a><h4>921</h4>kan stee , uden saa er at man opstaaer af Syndessvnen , det er , af Übodfcrrdighed , Sikkerhed og Ugudelighed . Derfor staaer der og , Av . G . , 2 , 38. : Omvender eder , saa stulle I faae den Hclligaands Gaver . Joh . 14,17 . : Verden kan ikke annamme <b>Sandheds</b> Aand ; men hvad er Verden andet end at leve ugudelig ? Og hvad er det ogsaa Herren siger Match . 7 , 20. : Paa deres Frugter stulle I kjende dem ? det er ikke andet end , at sande og falske Christne kunne kjendes af deres Levnets Frugter , men ikke deraf , at de get raabe : Herre , Herre . Thi de falske Christne skjule og besmykke sig med den rene Lårdoms Skin , saasom med Faareklcrder , alligevel de i Hjertet ere intet mindre end sande Christne . Vel bor ingen domme om Ens Lcrrdom ester Levnet , og indbilde sig , at Lårdommen derfor er vrang og falst , fordi den findes hos dem , der fore et stemt Levnet , saasom Gjendobere og Papisterne vrangeligen deraf domme ilde om vor , 3 oerdom , fordi saa mange iblandt os , ligesaavelsom iblandt dem , leve ilde . § 5. Thi det folger ingenlunde , at Lårdommen derfor maa være usand og . falst , fordi at Folk , som den have , ined deres ugudelige Levnet forholde sig ilde ; thi da maatte ogsaa Christus og A ' postlerne have lært det som var falskt og usandt , efterdi der og i deres Tid vare mange stemme Mennesker . Ikke dcsmindre er dog det onde Levnet en Prove paa Personen <b>selv</b> , hos hvilken det sindes , og vidner , om han er en falst eller en <b>sand</b> Christen , om han lever som han larer , om han troer rettelig , eller han <b>mor</b> anderledes og imod Troen . Om saadanne siger den Herre Christus , at de ere falske Christne , ufrugtbare onde Tra-er , hvilke derfor henhore til Ilden , Match . 7,19 . Omsider, kort at sige , da er det den sande Troe , der virker kraftig ved <b>Kjærlighed</b> , Gal . 5 , 6. , hvorved Mennesket bliver til et nyt Kreatur , fodes paa ny , og bliver forenet med Gud , hvorved Christus boer i os , Eph . 3 , 17. , lever og arbeider i os , hvorved Guds Rige opbygges i os , hvorved den Hellig-Aand renser og oplyser vort <b>Hjerte</b> . Derom vidne adskillige herlige Sprog i Skriften , som 1 Cor . 6 , 17. : Hvo som hanger ved Herren , eller holder sig til ham , han er een Nand med ham . Hvad er det at vare een

<h4>975</h4>übodfcrrdige og forhærdede Mennester evindelig forblive . Hvori sinder en Fader Glade ? i sine Bsrns <b>Kjærlighed</b> ; hvori glceoer sig en Brudgom ? : sin Bruds Kærlighed ( Es . 62 , 5 ) ; meget mere maa der komme usigelig Glade af Kjarligheden til Gud , vor himmelske Fader , som med . fin Munds Kys ( Hoi ' . 1 , 1. det er i Christo , ) paadet aller' kjarestc kysser os , og i ham , og ved den Hellig-Aands <b>Kjarlighed</b> kommer til os , og gjor Bolig hos os . Ich . 14 , 2 ) . § 5. Dog stal man ikke førstaae Guds Billede , som bestaaer i Ligedannelse efttr Gud , saaledes , som at Menne « stet skulde vare Gud ligt i alle Maader , og i lige saa stor Hellighed og Retfardighed , som Gud <b>selv</b> har , det vare langt fra ; thi Gud er übegribelig , umaalelig og uendelig i sit Vasen , Dyder og Egenskaber ; men Mennesket var alene « t Guds Billede og ' Udkast , og derfor ikke lignede ham anderledes, end som et Kreatur kan ligne sin Skaber , hvorom videre er talt i det første Capitel . Ikke desmindre var dog stig Lignelse en <b>sand</b> Lignelse , og Gud havde skabt Mennesket til sit eget klare og rene Speil , paa det , at naar Mennesket vilde vist vide hvad Gud var , saa kunde det ansee sig <b>selv</b> , og see Gud , som i et Speil , i sig ftlv , ja i sit <b>Hjerte</b> finde Guds Villede , og dette Guds Billede , med sine Egenskaber har varet Menneskets Liv og Salighed . Men det har den lede Satan misundt Mennesket , og brugt den allerstørste List og Tradskhed , som bruges kunde , til at førstyrre Guds Billede i Mennesket ved Ulydighed og Fiendfiab imod Gud . Ingen storre Svig og Underfundighed har nogen Tid varet bedreven , eller kan bedrives , end den Djævelen da brugte ; hvilket han gsorde deshellcr , efterdi det gik ud dervaa , enten han skulde evindelig være den ganske! menneskelige Scegts Herre , eller den menneskelige Slagt hans Herre . Og han har efter sin store List og Snedighed intet beqvemmere kunnet finde , til at bedrage . Menne « stene , og at stille dem fra Gud med , end det , hvorved han <b>selv</b> var falden og hvorveo han havde bedraget sig <b>selv</b> , som var , at ville vorde lige ved Gud . Saadan Begjæring indbildede han derfor vor første <b>Moder</b> , og gjorde det ved den deiligste og stjsnneste Ting , som var i Paradiis , paa den venligste og yndeligste Maneer og Maade han kunde .

<h4>1448</h4>Dm dell adle Dyd Ydmygheds Kraft . 393 bliver denne cedle Guds Kærligheds Lue desmere brccndende , og ved Troen opstiger til Gud , saa at den ogsaa i Gud i Christo elster alle Meuneffer for Guds store Kjærligheds Skyld ; tm ' den seer hvor hoilig den <b>selv</b> og alle Mennesker elsses i Christo . Og efterdi den er oploftet i Gud , ogindstuttetiGudsKicrrlighed , saa elsker den og alt hvad Gud elsker . § 3. Derpaa folger , at <b>Kjærlighed</b> glæder sig over alt Godt , som vor Næste vederfares , og sorger for alt Ondt ham <b>moder</b> . Og efterdi den er venlig og mild imod sin Næste , da fordommer den ikke strax sin Næste , som førseer sig af Skrobelighed , eller derfor ilde paataler ham , meget mindre opblcrser den sig imod ham , til at foragte og forhaane ham derfor ; thi den seer i en <b>sand</b> Ydmyghed sin egen Elendighed og Blusel , og at den <b>selv</b> sidder nedscenkt i lige eller fast storre Ulykke og Synder ; og , om den staaer , da kjender den vel , at den ikke af sig <b>selv</b> , eller ved sig <b>selv</b> har kunnet blive bestaaende , men at den er veo Guds Naade opholdet . Den ydmyge <b>Kjcrrlighed</b> dommer og straffer altid sig <b>selv</b> , for den dommer og straffer Andre , og naar den seer sin Næstes Ulykke , gaaer den ind i sit eget <b>Hjerte</b> og bejamrer sig <b>selv</b> . Thi den seer i sin Næstes Fald sine egne Feil , Synder , Jammer og Elendigheder . Wed Ydmyghed bliver et Menneske styrket i Guds Kundskab og Haabet . Thi naar et Menneske beskuer sin egen Vanvittighed , Blindhed og Daarlighed i Guds Sager , da takker det Gud for sit Ords Aabenbarelse , og tager det desmere til <b>Hjerte</b> , og naar det betragter , at al dets Formue og Kraft er intet , da styrkes det i Haabet , saa det scrtter detpaa Gud alene . Bed Ydmyghed bliver ogsaa et. Menneste yndeligt og behageligt i sit udvortes Levnet . Thi denne Dyd lader etMenneste ikke være trcettesyg og klammerkjcrr , men foielig , lemfældig og from mod alle . Den tilsteder En ikke at være hevngjerrig , og at betale Ondt med Ondt . Den betaler ikke Skjcendsord med Skjcendsord , men lader stig Storm kun snurre over , som vi see vaa Davld , der i den Mening sagde : „ Jeg maatte være som en Dov , og vll lkke hore , og som en Stum , der ikke oil oplade sin Mund ; ja jeg maa være , som en Mand der ikke horer , og som lM mgen Overdeviisning i sin Mund . " Ps . 38 , 14 , 15. Derved bliver saadant troende og ydmygt Menneste et levende Chri- Christus <b>selv</b> lever . En Troendes Ydmyghed ' ) " ) Udi Troens Kraft og Lysj ikke for sig alene , eller af sin egen Kraft .

<h4>3931</h4>1092 En Gjenlagelse af den Lårdom oin den Ord ind , hvorved baade Legem og Sicel bliver gledet og frydet ? Ja den svarer paa alle vore Tanker i Aand og <b>Sandheds</b> Hvad sagde Herren nl Pauws , da han bad ? " Min Naade er dig nok ; thi min Kraft fuldkommes i Ekrobeligyee . " <b>Mor</b> . 12,9 . Derfor maaChristi Ord ikke være udenfor os , og mange tusinde Miles Vei borte fra os , og just det er det , Taulerus kalder " det evig ? Faderens Ord , sm ; taler i vor Sjæl . " Der er jo ikke enhelligTanke udi os , som jo den Helligaand forudseer og førstaaer ? P5.1Z9 , 2. OgnaarGudiNod og Angester erindrer os om sin Forjættelse , da han siger : " ssrygt dig ikke , thi jeg er med dig ; ste ilke om til Andre , tdi ! eg er din Gud " ( E 5.41 , 10. ) , hvor taler han da med os , udenfor eller i vore fjerter ? Jeg siger dig : Bil Gud ikke ogsaa tale sit Ord , som han haraabenbaretos i den hellige Skrift , i dig , med dig og til dit <b>Hjerte</b> , saa ffa ! du ikke have stor Nytte af Guds Ord . I hsie Anfegtningcr udviser det sig . om Guds Ord maa være levende i vore Hjerter eller ikte ; , chl drager Gud det sammesK raft etlidet tilbage fra Hjertet , o Gud ! hvad kunne da alle Boger og Konster udrette ? Hvo kan noksom takke Herren for hans Ords Kraft , naar detvirkcr i vore Hjerter ? Hvo erkjender dette ? Hvad har Gud talt med de hellige Martyrer i deres Anfegtning og Dødsmarter ? Hvad talede han med Stephans ? Hvor stede det , da han saae den herlige Aabenbarelse ? Av . G . 7,50 < 56 < Sandelig, dersom du ikke faaerGud at hore <b>selv</b> tale sit i Ord , og heier du ikke Guds Ord , som af Guds Mund , og tager det ikke til <b>Hjerte</b> , , XL . saa staldu aldrig kunne fole nogen Livskraft deraf . Det 16 , i ' ! og 18 Capitel . Verdens <b>Kjærlighed</b> hindrer den Helligaands Virkning i os . " Jeg vil ikke fordriste mig til at tale noget om de Ting , som Christus ikke har virket ved mig , " siger Paulus Rom . 15 , 18. Men stal Christus vort eneste Hoved virke ved og i os , som sine Lemmer , saa maae vi ikke forhindre hans fjerning i os , og leere vel at kjende , hvad Gud virker i os , fra andet , paa det oi ieke stulle Satans og vort Kjøds ( gerninger . Den Helligaand virker en gudelig Bedrovelse i os over Synden , og tilsteder ikke noget Menneske " at befalde sig <b>selv</b> , " Rom . 15,1 . ; han betager os al vor egen Roes , al egen Retfærdighed , og lader densammeforvisnesometßlomster,ogvirkercnmeolidendeßarmhjertighed

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016102148034'target=_>Luther, Martin, 1858, Dr. Martin Luthers Prædikener over alle Søn- og Fest-Dages Evangelier og Epistler samt Lidelses-Historien</a><h4>6818</h4>ei straffe det Onde ? Hvad vilde der vel komme af det , at man stulde taale alle Ting , stjule og fordrage alt det Onde , som steer ? Derved vilde man jo <b>selv</b> give Aarsag til alt Ondt , saa at tilsiost Ingen kunde leve i Ro for den Anden ? Svar : Det er ofte sagt , hvem det tilkommer at blive vred og straffe , og hvorvidt , og paa hvad Maade . Thi det er <b>sandt</b> , Ovrigheden i Landet og enhver Huusfader stal vredes , straffe og forhindre det Onde ; ligeledes , en Prcest og en Lcrrer iftlge deres Kald , ja , ogsaa enhver Christen stal formane og straffe sin Næste , nåar han feer ham synde ( ligesom den ene Broder i Huset straffer den anden ) . Men det er noget andet at blive vred over det Onde , og af Embeds-Pligt at straffe , end at hade og være hevngjerrig , eller snste Ondt over Nogen , og ei at ville tilgive . At blive vred og straffe , nåar man feer sin Næste at synde , det er ei imod Kjærligheden; thi den rette Kjærligheds Natur er ogsaa saaledes , at den ei gjerne seer sin Næstes Synd og Skam , men gjerne vil have ham forbedret . Saaledes ogsaa , nåar en Fader og <b>Moder</b> see at deres Barn er gjenstridig og ulydig , saa komme de vel strax med Riset , men forstode dog ikke deres Barn eller hade det ; tvertimod soge de at forbedre det , og nåar det er straffet , kaste de Riset bort ; og saaledes maa ogsaa du ( siger Christus ) straffe din Broder , see suurt og med Vrede paa ham , at han kan vide og tilstaae , at han har gjort Uret ; og dersom han da ei forbedrer sig , saa maa du melde det for Menigheden ; men du stal derfor ei hade ham , og stedse bære Forbittrelse og Vrede mod ham i dit <b>Hjerte</b> . — Thi den rette <b>Kjærlighed</b> maa , som sagt , ei være doven og told , saa at den ei stulde agte sin Næstes Synd og Fordcrrvelse ; men den maa tænke paa , hvorledes den stal kunne

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111303008'target=_>Nohrborg, Anders, 1859, Det faldne Menneskes Saliggjørelses-Orden</a><h4>1029</h4>sit <b>Hjerte</b> derover ; ikke fordi han er bleven anseet vcerdig til og har Anledning til at vcere et Redskab for Guds Forsyn til sine Medbrsdres Tjeneste ; nei ! thi dette er ogsaa en <b>sand</b> Christcns Glcede ' ) ; men fordi han tykkes sig at verre stor , uundvcrrlig , nsdvendig i Verden , og i Andres Tanker og Omdsmme om ham ; uagtet det er vist , at Andre aldrig scette saa stor Priis ftaa hans Gaver , Duelighed og Gjerninger, som han <b>selv</b> : thi Andre have ikke saa stor <b>Kjcrrlighed</b> til ham , som han har til sig <b>selv</b> , og det er KjMigheden , som seer igjennem Fingre , og seer ingen Feil og Mangler , men nndstylder dem " ) . Derfor hcender det ogsaa som oftest , at den Allerudneligste , for sin Egenkjcerligheds Skyld , sinder det storste Behag i sig <b>selv</b> , sin Forstand og sine naturlige Gaver . § 9 . Lykkens Gaver . ere ogsaa en Anledning til Hovmod hos Verdens Bsrn . De hovmode sig over deres Fodsel , saa at den , som har havt den Lykke , ved Guds Forsyn , at fs « des af fornemme Forceldre , og saa at sige under hsiere Tag og inden sterre og betrukne Vcegge , han indbilder sig naturligen at verre bedre og cedlere end Andre , hvem han betragter med Foragt, som Skabninger af et ringere Slags . Andre hovmode sig over Klceder og Stads , som efter udvortes Anseelse adskilte dem fra den almindelig Hob ^ ) . De ere enSlcegt , siger Sa « lomo , hvis <b>Oine</b> ere heelt hoie , og hvis Dienlaage ere oploftede ^ ) . Men kunde vi , som Gud , see ind i deres Hjerter , saa skulde der ikke sees saa megen Prydelse ; thi at vcere klcedte i Retfcerdighed , det ligger i Almindelighed ikke mere de Christne saa hsit paa Hjertet . Det Udvortes er dem vigtigere ; og mangen Cn anvender al sin Omtanke , Omsorg og Evne , alle sine Begjæringer , ja hele sin Naadetid paa lidet Andet , end Paa udvortes Prydelse, Klcrdedragt og paa at opfinde og efterabe nye <b>Moder</b> . Nogle hovmode sig over

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009091403021'target=_>Luther, Martin, 1864, Dr. Martin Luthers Udførlige Udlæggelse af første Mosebog</a><h4>57</h4>Braadde , som have cegget ham til Fortvivlelse og Vantro . Ligesom ogsaa Jobs <b>Hustru</b> bebreider sin Mand , idet hun siger : holder du endnu fast ved din Fnldtommenhed ? velsign Gud , og do ! Job 2 , 9. Dette havde man ogsaa kunnet sige til Isak . Thi den , som forhen var foragtet og blev forstodt ud af Hufet , faaer den guddommelige Velsignelse , som er langt rigere og storre , end baade Isak og Jakob nogensinde fik den . 20. Saaledes er dette underfulde Beretninger og give de kjodelige Mennester Anledning til store Forargelser . Thi alle Guds Gjerninger stride lige imod Forjettelsen , hvilken dog alligevel forbliver aldeles sanddru og uforandret . Men dette forarger Folk , at den er usynlig , saaat den forhales og ganste vendes om . 21. Derfor stal man lere Guds underfulde Raad , hvormed han styrer sine Hellige , og de Gudfrygtiges Hjerter stulle venne sig dertil . Har dn en gnddommelig Forjettelse , da vil denne Forjettelse desto mere skjules for dig , forhales og vendes om til det Modsatte , jo mere du elskes af Gud . Thi hvis han ikke elstede dig saa hoit , vilde han ikke lege saaledes melt dig , det er , han vilde ikke saaledes forhale sin Forjettelse og Hjelp og vende den om til det Modsatte . 22. Thi det er det sikreste Tegn paa et faderligt <b>Hjerte</b> , der ligesom brender af den store <b>Kjerlighed</b> , han nerer til dig . Saaledes gjor ogsaa en legemlig Fader , som elster sit Barn : han leger med det , lover at give det Gaver , men forhaler det og anstiller sig , som om han ikke vilde holde sin Forjeltelse . Dette gjor han , fordi han elster Barnet hoit , og vil rigelig give det , saafremt Barnet ogsaa bier og kan taale Opsettelsen . Saaledes leger ogsaa Gud med os ; men vi blive snart vrede og utaalmodige , <b>selv</b> for en liden Opsettelses Skyld . Thi det , som forjettes os , ville vi enten strax have for vore <b>Oine</b> , eller ogsaa ville vi faa det paa en anden Maade og Vis , hvad det saa end stal vere for en Maade .

<h4>178</h4>124. Hidtil er det bleven sagt , hvorledes Gud skjuler sig , jn ganste er bleven borte for den hellige <b>Kvinde</b> , jn , ogfnn for Ifak <b>selv</b> og for Abraham , dertil for Alle , fom vare i den samme Menighed , saaat hnn maatte lobe og tåge sin Tilflugt til Patriarken Sem og hans Menighed og soge Hjelp og Raad der . Men nu beskriver Moses , hvorledes denne tnbte Gud atter findes og lader sig se , os nemlig til Exempel , sorat vi ikke stnlle forsage i vore Anfegtelfer , nåar Bonnen holder sig til den visse Forjettelse , men stulle vere visse paa , at Gud visselig maa vise sig , lade sig se og give Svar , om san end Himlen og alle Skabninger stulde true med Fald og Undergang . 125. Men hvorvel nu denne Venten er saare besverlig , stulle vi dog vere saaledes underviste og lerte nf den Hellig-Aand , nt vi vide , nt Gud visselig maa hore vor Bon , efterdi han nemlig ikke kan lyve . Nu havde hnn forjettet Isak Velsignelsen ; men ved denne Forjettelse forefnldt mnnge Besverligheder og Forhindringer , snnnt et svagt <b>Hjerte</b> vel kunde tvivle , om vel ogsaa Forjettelsen var <b>sand</b> , og hvorvel Rebekka ikke havde nogen synderlig Forjettelse , bad hun alligevel , og nanr nu Bonnen er ret og <b>sand</b> , som Rebekkas Von uden Tvivl var , maa den dog nodvendigvis holde sig til den guddommelige Forjettelse . 126. Thi hvo skulde vel af sig <b>selv</b> vere saa dristig , nt hnn havde oploftet sine Hender og <b>Oine</b> til Gnd , hvis han ikke var vis pnn Guds Villie ? Derfor holder nl Bon sig til Guds Bud og Forjettelse . Thi i det andet Bud staaer Guds Ord , som lerer os at bede , nåar nemlig Gud siger 2 Mos . 20 , 7. : du skal itte tåge Herrens , din Guds , Navn sorsleugelig . Ligesaa Ps . 50 , 15. : tald paa mig paa Nodens Dag o . s . v. Men nåar nu Bonnen er skeet , stal du vide , at du ligesaalidet maa tvivle paa Bonhorelfen , som du kan eller stal tvivle pnn Forjettelsen. Thi Bonnens Grundvold er Forjettelsen , ikke vor Villie , ikke vor Verdighed eller vor Fortjeneste . Og pnn denne Lere hnve vi et herligt Exempel i Rebekkas Bon , som efter snn mange Forhindringer tilsidst underfuldt bonhores . Thd da hun i Begyndelsen bliver frugtsommelig , tvivler hun ikke paa , at hun nu er bonhort og hjulpen af Gud ; men denne Tro og Tillid forhindres snart , ved en ny Fare , som hnn ingenlunde havde ventet . Thi Bornene stodes tilhobe i hendes Liv , saaat hun atter bringes til Fortvivlelse . Men da hun nu er paa det Hoieste forskrekket , avreiser Gud hende atter med Ordet og viser , at hun i

<h4>675</h4>sonderriver det og omgaaes ilde dermed , men alligevel forbliver et godt Trce , faaledes var Ifak ogfaa ligesom et Trce , der er plantet ved Vandbakke, som giver Frugt i sin Tid , Ps . 1 , 3. Thi ved at vcere saa taalmodig og bcere den Uret , som <b>moder</b> ham , aabner han Naboernes <b>Oine</b> , bcvceger og blodgjor deres Hjerter , saaat de erkjende og bekjende den Synd , som de havde begaaet mod ham , og tcenke , hvor stor Vold og Uret de havde tilfoiet ham , hvormeget og hvor übillig han hidtil havde maattet lide af dem uden nogen egen Skyld , eller uden sin fromme , cerbare Hustrues og hele Tyendets Skyld , hos hvilke Alle de til enhver Tid havde mcerket synderlig Venlighed , Mildhed og Gjcestfrihed, ligefom de ogfaa hos hans Tjenere og Piger havde feet , at de havde gode Sceder og vare ret og vel underviste og lcerte ; derfor erkjende de , at Gud er ncervcerende med sin Velsignelse hos et saadant hcederligt og gudfrygtigt Hus . Saaledes bringer Isaks Taalmodighed og Venlighed Fienderne til Bod og Selverkjennelse , saaat de nu revse sig <b>selv</b> og give sig Skylden for at have fordrevet den fromme , hellige Mand og hans Hus . 277. Og faaledes stemmer dette smukt overens med Pauli Ord Rom . 12 , 20. , som han har tåget fra Ordfpr . 25 , 21. 22. : dersom din Fiende hungrer , giv ham Mad ; dersom han torstcr , giv ham Dritte ; thi nanr du det gjor , samler du gloeude Kul paa hans Hoved . Thi hvor forbittret et Menneskes <b>Hjerte</b> end er , saa kan det dog blodgjores og forsones , hvis det itte er ganste forhcerdet og forstokket i sin Ondstab, saaledes som Pharaos <b>Hjerte</b> var . Men hvis det af mennestelig Svaghed eller Uvidenhed , eller ved falst Angivelse af andre Folk , eller ogfaa af retfcerdige Aarsager er bragt i Vrede , kan det dog saaledes blodgjores ved Velgjerning og Taalmodighed , at det oplcrndes mod sig <b>selv</b> og erkjender hans Godhed og Venlighed , som det holder for sin Fiende , revser sig <b>selv</b> og tcenker : hvorfor har du dog krcenket denne fromme Mand og gjort . ham Skade ? hvorfor har du forfulgt ham , forn er faa uskyldig ? Det er Kullene , som scettes i Glod og begynde at brcende ved saadan Taalmodighed , Venlighed og Velgjerning . 278. Paa denne Maade bevceger og overvinder Isak ogsaa Kongen , idet han forbliver i Tro , Haab og <b>Kjcerlighed</b> og er gjcestfri , faaat han itte alene blev forsonet og forligt med ham , men derved ogsaa omvendes til den sande Gudsdyrkelse og den rette Gudserkjendelse . Thi uden Tvivl Horte han Prcedikenerne og gik til Isaks Pauluner for at lcere Guds Ord af ham . Saaledes vandt Gud Kongen og mange andre Mennester ved det Ord , forn Ifak lcerte , og bragte dem til den rette Erkjendelse . Thi han har givet os sit Ord , ikke alene for vor , men ogfaa for Andres Skyld , nemlig foråt vi stulle meddele andre Folk det . 279. Paa denne Maade udbredes den behagelige Lugt baade af Isaks rene , sunde Lcere og af hans uskyldige Liv hos Kongen og Fyrsterne, saaat de vel sagde : ak hvorfor fordreve vi dog denne fromme , hellige Mand , som baade lcerer ret og herlig og dertil forer et helligt Liv ; hvis <b>Hustru</b> er en saare from <b>Kone</b> , dertil velgjorende , mild og prydet med allehaande Dyder ! Og saaledes bleve Menneskenes Hjerter omvendte i hele Riget og Nabolaget . Vcer derfor taalmodig , hvo du end er , som lider Vold og Uret af andre Folk , og bi paa Herren ! Thi alt dette ledes og styres af Gud , medens Isak sidder stille og flet itte foretager sig Noget mod Kongen , heller ikke slutter noget Forlig

<h4>790</h4>Uforsigtighed stoder an og feiler , saa pleier dog Gud at forudse og rette paa Saadant , foråt Feilene stulle tilhylles , og Sagen dog tilstut faa en lykkelig , god Udgang . 62. Jeg ved , at jeg ogfaa ofte har gjort mange Ting taabelig og meget uforsigtig , ja faaledes , at jeg vel har tcenkt : hvorfor har Gud kaldet mig til Prcesteembedet , da jeg jo dog itte har saa megen Klogt , saa megen Naadsnildhedx og Forstand , som der krceves til dette store Embede ? Thi om jeg end har gjort Alt af et godt og enfoldig / <b>Hjerte</b> , dertil ogsaa med Flid og en god Iver , saa er der dog ofte hcendt meget urimeligt Toi og nogle Feil ere ogsaa komne med iblandt , saaat det har seet ud til , at baade Himlen og al Verden vilde styrte sammen . Da maatte jeg falde paa mine Kncr og bede Gud om Hjcelp og Raad , han som er mcegtig og kan bevirke , at <b>selv</b> et haardt , grueligt Anstod eller Uheld maa vinde en god Udgang , medens vi sove , ligesom han stabte Eva , medens Adam sov ; thi da tog han et af haus Ribben og lullede med Kjod istcdet derfor og byggede en <b>Kvinde</b> af Ribbenet , i Mos . 2 , 20. 21. Her kunde man sige , at Gud maatte have Silkefingre, siden han med saa ringe Moie og saa let kan ndfore en saa stor Gjerning . 63. Netop paa denne Maade styrer han ogsaa sine Hellige . <b>Selv</b> om de en eller anden Gang have faret vild eller feilet grovelig , og uforsigtig begaaet en stor Daarlighed , hvoraf der kunde komme utallig megen Ulykke , faa kan han dog gjore en glad og lykkelig Udgang paa det Altsammen. Saa kraftig er Troen og Bonnen ; ja . Troen er i <b>Sandhed</b> almcegtig . 64. Saaledes bliver Isak ligesom med sit Vidende og i sine aabne <b>Oine</b> bedragen ; thi han siger <b>selv</b> : Rosten er Jakobs Nost , meu Hamderne ere Esans Hamder , og lader sig dog bedrage . Ligesom man ogsaa i Hedningernes Boger kan se , at Herrerne bedrages af sine Tjenere , som hin siger i Komedien : var jeg itte saa ganste uforstandig og raa forn en Sten eller Klods , saa kunde jeg mcerke Bedraget . Saaledes , siger jeg , handler Gud med sine Hellige eller Troende , at de itte engang kunne mcerke eller forståa , hvilken taabelig eller uforsigtig Sag der er paafcerde . 65. Derfor stulle vi flittig tcenke paa dette Exempel , ved hvilket vi paamindes om , at Troen har faa stor Kraft , at den gjor dem , forn kunne hore og fe allerstarpest , blinde og dove , faaat de bedrages , stjont de baade hore og fe . 66. Og saaledes kan man nu svare paa dette Sporgsmaal , at Rebekka vel handlede uforsigtig ; men det guddommelige Forfyn ledede hende for hendes herlige , store Troes Skyld , saaat Faderen , Isak , hvorvel han Horte Jakobs Stemme , dog blev bedragen , fordi han havde faa stor Tillid og var saa ganste tryg . 67. Et vantro Menneste bedrages ikke lettelig ; men hvo der troer Enhver , han bedrages ogsaa lettelig , som Salomo siger Ordspr . 14 , 15. : en Vanvittig troer . Alting ; men eu Klog agter Pan sin Gang . Derfor maa man ikke give sig ikast med dem , som man vil stusse , paa den Tid , nåar vi vide de vaage og give Agt ; men forst maa man fjerne al Mistillit) fra dem og maa faa dem til at tro os . Man maa stedfe opretholde deres Hjerter med Haab og faa dem til at tro , at de itte have noget Ondt at vente af os , ligefom Tjenerne Davus og Syrus i Komedierne forståa at bedrage sine Herrer saa listig , at de lutte Oinene ,

<h4>1294</h4>28. Dernest tjener dette os ogsaa til Trost , nåar de hellige Patriarkers Exempler foreholdes os , ikke alene hvad angaaer de hoie og heltemodige Dyder , men ogsaa i de simple og ringe Gjerninger , ja i Husholdningens ureue og foragtelige Gjerninger , vaadet vi ikke stulle forsage eller mene , at vi ere foragtede og forsmaaede af Gud , nåar vi belesses med disse huslige Gjerninger ; men vi stulle vide , at Alt helliges ved Ordet og Troen . 29. Hvorvel Verden ikke ser denne Hellighed , men nåar den horer , at saadanne simple , almindelige Gjerninger fortelles om de helligste Mennester , saa mener den , at det er idel forgjeves Arbeide , og at man tilbringer den kostbare Tid flet med at lese saadanne Historier . Thi den er ikke verdig til at se Guds 3 - Ere og Herlighed , som strevet staaer : bort med den Ugudelige , at han ikke stal se Guds Herlighed . Alene de Troende se og forståa Guds Gjerninger ; derfor er Saadant for vore , ja for Guds <b>Oine</b> kostelige Gjerninger . 36. Og du behover ikke at tenke paa eller undre dig over , hvorfor den Hellig-Aand har Lyst til at beskrive saadanne simple og foragtelige Gjerninger , men hor , hvad den hellige Paulus siger Rom . 15 , 4. : hvadfomhelst tilforn er strevet , er tilforn skrevet os til Lmdom , vaadet vi skulle have Haab formedelst Taalmodigheden og Strifterues Trost . Derfom vi fast troede , faaledes forn jeg , hvorvel i Svaghed , troer , at den Hellig-Aand <b>selv</b> og Gud , alle Tings Skaber , er den rette Forfatter af denne Bog og til saadanne simple , foragtelige Ting , der tykkes Kjodet at vere usle og ringe , da vilde vi have den storste Trost deraf , som St . Paulus siger ; ja , vort <b>Hjerte</b> vilde rose sig deraf og vere stolt af , at Gud nedlader sig til endnu saaledes at komme disse Patriarker ihu og ikke glemmer dem , og at han ikke alene vil , at deres ridderlige Dyder stulle roses , men ogsaa de urene , simple Gjerninger , som med denne Beskrivelse stulde prydes ligesom med Guld og Wdelstene og foreholdes al Verden , for faaledes at udbredes , lefes og erkjendes. Derfor tjene alle Ting de Troende til Gode ; ja , hvad der ogfaa i Forveien var ringe , urent og foragtet , maa tjene dem til 2 Ere , idet de fe , at Gud har Lyst til saadanne ringe Ting , saaat han ikke alene vil besynge og lovprise de Helliges store og herlige Dyder , men ogsaa deres ringeste Gjerninger . Thi de ere Guds Gjerninger ; derfor har Gud Lyst og Glede af sine Gjerninger , hvad enten de ere store eller smaa , og det til vor store Trost , som der staaer Ps . 147 , 11. : Herren har Behagelighet » til dem , som ham frygte , og som David siger Ps . 56 , 9. : du har talt min Flugt ; tag min Grand i din Flaske ; har du Me Tal dervaa ? 31. Hvorledes ? har da Gud intet Andet at bestille end at telle Davids Taarer og hans Flugt ? Har han ikke ellers nok at bestille med at regjere Verden ? eller med at hore paa Englenes Chor , der uden Ophor love og prise ham ? Hvad Forunderligere kan der dog siges ? Og dog er det <b>Sandhed</b> , og Gud sorger ogsaa for at telle Davids Taarer og hans Flugt , som der staaer Ps . 9 , 13. : den , som hlrvner Blod , kommer dem ihu , han har Me glemt de Elendiges Skrig . Ligedan Ps . 116 , 15. : det er tosteligt for Herrens <b>Oine</b> , nåar hans Hellige do . Ja Mofes siger til Pharao 2 Mos . 10 , 26. : endogsaa vort Ftr stal gaa med os ; der stal Me en Klov blive tilbage ; ikke alene Mend , <b>Kvinder</b> , Born og Kveg stal drage ud af Egypten ; men Alt , hvad vi have , endog den mindste Klov , ville vi have med os .

<h4>3087</h4>44. Paa denne Maade har jeg udlagt denne Text , nemlig om det faderlige <b>Hjertes</b> Tilboielighed og <b>Kjerlighed</b> , at nemlig Jakob efter faa megen og mangehaande Slags Angst , Kors og Gjenvordighet » alene holdt sig til denne Son , som var fodt af den rette og fornemste <b>Moder</b> , som ogsaa var omkommen under store Smerter under Fodfelen . Derfor havde han al sin Trost , Lyst og Glede af denne Son . Thi han alene kunde troste den gamle Fader i saa megen Angst og Kummer , medens de andre Sonner bekymrede og plagede ham , ja vel daglig voldte ham ny Bekymring og ligesom ophobede den . 45. Men Faderens Moest , Omhed og <b>Kjerlighed</b> blev ogsaa altid storre derved , at han hos Joseph saa en synderlig Gudsfrygt og allehaaude Dyder . Thi lofeph var ogsaa i <b>Sandhed</b> vis , tugtig , from , gudfrygtig , dertil prydet og begavet med hoie og fortreffelige Forstandens Gaver . Derfor er han ogsaa sin Faders Oiesten , Lyst og Glede . Thi saaledes pleie Foreldre at bere sig ad : om En end havde tyve Born og blandt dem Alle kun een , som var from og lydig , vilde Faderen Me vere anderledes tilsinds mod dette fromme Varn , end om han Me havde havt flere ; de ugudelige og ulydige vilde han gjore arvelose eller dog ialfald staffe sig af med. dem , Me sorge for dem og ganste opgive dem . 46. Men nu er der eudmt en Vanstelighed paa dette Sted i sproglig Henseende , nemlig angaaende Kjortelen af adftillige Farver . Hvad dette har veret for et Kledningsstykke , bekjender jeg lige ud , at jeg ikke lan vide . Vi have i vor Oversettelse gjengivet det ved : en Kjortel af adftillige Farver , forn det da ogfaa ellers almindelig er bleven udlagt . Lyra siger , at det stal have veret V6Btiß 861 - ica , ett Silkekledning. Burgensis gaacr stiltiende forbi dette Sted , siger Intet derom . Men Andre mene , at det har veret et Kledningsstykke af mere end eet Slags Traad og paa den Maade broget , idet Rendingen har veret rod , og Indslaget himmelblaat , faaledes at de to Farver for <b>Oiet</b> blande sig sammen og blive brogede . Men denne Mening tykkes mig ikke

<h4>3183</h4>Sichem ; eller ogsaa drog de af den Grund til et andet Sted , at de ikke lengere kunde taale losephs Nerverelse og Omgang , idet de vare optendte mod ham med saa stort Had og Avind , at de ikke engang kunde tale venlig eller fredelig med ham , ja , de gad vel Me engang se ham for sine <b>Oine</b> . Thi det kan vel hende , at En af Fiendstab og Had til en Anden flyer for ham , som der siges om Kam 1 Mos . 4 , 6. : da blev Kam meget vred og hans Ansigt falmede ; han kunde ikte se paa sin Broder Abel i Venlighed ; men strax han saa ham , forbitredes han og viste ved sit Ansigts sure Mitter , at hans <b>Hjerte</b> brendte i bittert Had . Den samme Grund kunde maaste ogsaa drive disse Brodre til at soge Gresgang paa et andet Sted ; men Joseph blev hjemme hos sin Fader , medens de Andre saaledes flyede for ham og uttdgik Synet af ham . 133. Dernest fe vi fremdeles , at hans Slegt alligevel endnu havde Sichem iude , siden de drog derfra med Kveget . Thi efter Sichemiternes Mord kom Jakob med sine Sonner i deres Sted , og Sichemiternes Enker og hvem Anden der var bleven tilovers af Folket , forblev uden Tvivl hos Jakob der i deres Fodelattd , og det er min Tro , at ogfaa Naboerne i Kanaans Land bleve forligte og forsonede med dem , nemlig paa Grund af Suogerstabet med Ittda , som fortelles Kap . 38 , 2. Derfor fik de ogfaa <b>Kjerlighed</b> til de Andre , forn vare tugtige og fromme . Men de onde Misdedere , Simeon og Levi , Sichemiternes Mordere , have de udeu Tvivl afstyet . 134. Men dette Sted indtog dernest Joseph med sine Efterkommere, just det samme Sted , hvor han i Virkeligheden blev slaaet ihjel af sine Brodre . Thi sidenefter skjenkes dette Land ham af hans Fader til en Besiddelse , hvilket Jakob jo Me havde kunnet gjore , hvis han Me havde havt det inde som sin Eiendom . Saaledes drog de da fra den Ager , som de have forvagtet , til Sichem , til sin egen Gresgang . Og atter , nåar de Me lengere <b>sandt</b> Gresgang der , droge de tilbage til Hebron igjen . Thi de kunne og ville Me lengere forblive hos den stjeudige , onde Forreder , forn vilde , at de Andre skulde boie sig for ham . 135. Endvidere kaldes efter dette Sichems Land Israels og Ephraims Rige ogsaa losephs Rige , og dets forste Konge leroboam var fodt af denne Slegt . Thi de saa hen til Prophetiett , i hvilken sidenefter Herredommet over dette Sted skjenkes Joseph , idet Jakob beholdt dette Sted , hvorvel det var erobret ved hans Sonners grumme Voldsgjerning , idet Gud tillagde ham det og saaledes vendte hans Sonners onde Daad til det Gode . Thi saadanne timelige og verdslige Goder , hvorom Gud intet Synderligt har forordnet eller befalet , ere kutt . tilfeldige ; Gud uddeler dem til hvem han vil . Hvo der har det , han har det . Jakob havde ingen Forjettelse eller Befaling om , at Sichems Indvaanere saaledes skulde slaaes ihjel ; men da Hemor og hans Son vare blevne drebte , beholdt han Landet med Naboernes Samtykke . Saaledes gaacr det endnu til paa en tilfeldig Maade , at Mange saa Eiendomms , hvorom de dog ingen synderlige Forjettelser eller Befalinger have af Gud . 136. Hvad der nu herefter folger i Texten , er ikke vansteligt at forstua , hverken hvad Meningen eller hvad Svroget angaaer . Mofes beskriver den gamle Fader Jakobs Omsorg for Husvefenet ; <b>selv</b> var hatt gammel og tung paa Grund af sin Alder , saaat han ikke personlig

<h4>3621</h4>Sine . Her tiltrengtes sandelig en fast Tro , en utrolig Styrke og Taalmodighed . 11. Det Ord , forn hans Fader , Jakob , havde indplantet i Drengens <b>Hjerte</b> , var rent og faare frugtbart og bragte rig Frugt , forn der staaer strevet Lue . 8 , 15. En Anden , forn havde veret paa lofephs Alder , var enten bleven overvunden af Utaalmodighed eller Hjertets Kummer , eller havde benyttet sig af Anledningeu til at drive Utugt . Men hos Joseph ser du intet Saadant ; hos ham fremlyse og aabenbares mange stjonne Dyder paa een Gang , saadanne som kunne finde Sted og udsnes i denne vor Natur . Der er , siger jeg atter , intet andet Sted noget Exempel , som kuude sammenlignes hermed . Derfor have heller Me de andre Brodre med den Flid eller Frugt hort paa Faderen , hvorvel det var den samme Lere og det samme Ord , som Jakob uden Tvivl bestandig med Flid og Trostab har foreholdt sit hele Hus . Og dog blive de Fader- og « Modermorders , ja rene Djevle . Joseph alene er en Engel ; han alene antager sin Faders Ord ; han fatter Tro , Haab , <b>Kjerlighed</b> og holder fast derved , saaat han Me lader sig det fratage , hverken ved Satans Magt eller hemmelige List . 12. Det er en stor Jammer , at han rives sin Fader saa langt bort fra Oinene ; hnn har vistnok Me forladt sin Faders Hus uden stor Smerte , Klage og Grand , baade over sig <b>selv</b> og over Faderen ; Gud tier stille og lader , forn om han Me ser denne Jammer ; Joseph sukker og striger og ser ud , som om han var ganste forkastet ; Alt er forbi . Men hvorfor fortvivlede han Me i sit <b>Hjerte</b> og rctabte : jeg ser , at jeg er forladt af Gud og Mennester , og at jeg uden min Brode betynges med faadan übillig Ulykke , jeg forn dog har holdt min Fader i Mre og adlydt ham i alle Ting , — faaledes forn Fortvivlede pleie at klage med Heftighet , ? Der er ingen Tvivl om , at ogsaa hans <b>Hjerte</b> har lidt og udstaaet saadanne hactrde Slag og Storme . Thi han var en Ingling paa 17 Aar , og vi maa Me tenke , at han var Stok eller Sten , saaat han slet ikke skulde have solt denne store Ulykke , da han blev kastet i denne Fristelse til Fortvivlelse , Vrede og Fiettdstctb mod Gud , dertil ogsaa cmfegtedes med Utugt og tilslut maatte taale den ynkeligste Treldom , idet han blev tvungen til at tjene fremmede og übekjendte Mennester forn den ringeste livegne Trel , dertil var forrctctot og forstodt af sine kjodelige Brodre . Se til , om Me dette er en forskrekkelig og utactlelig Elendighed , hvormed vor Ulykke ikke paa langt ner kan sammenlignes ! 13. Saaledes arvede han nu i <b>Sandhed</b> sin Faber , de Trengsler nemlig , som Faderen havde lidt . Thi hvorvel ogsaa luda havde et Me lidet Kors , er det dog Intet mod dette Kors at regne . Joseph er endnu meget ung ; det er for tidlig for ham ; han er endnu kun 17 Aar gammel og maa dog savne al den Bekvemmelighet » , Fordel og Hjelp , som er fornoden for en faa ung Alder ; han kommer til et fremmed , übekjendt Land , til ugudelige , afgudiste Mennester , hvor han hverken havde Lerdom eller Exempel , hvorved han havde kunnet undervises eller styrkes , men maa for sine <b>Oine</b> se lutter Nacched og de skjendigste Exempler ; og i det Indre har han vel ogsaa solt og erfaret Djevelens Lift og gloende Pile . 14. Og dog alligevel herskede i hans <b>Hjerte</b> det Ord , forn han havde hort af sin Fader ; ved det holdt han fast . Min Fader , tenkte

<h4>3690</h4>med stor , utrolig Taalmodighed og hoit Mod har overvundet al Ulykke og Besverlighed i sin Elendighed , hvortil han var bleven stebt hen af sine Brodre og Frender , idet han tillige maatte vere berovet Forstefodselens store 2 Ere og Forhaabninger og bo blandt rnae , ctfgudiste Mennester ; men ved alle disse Besverligheder kunde dog hans <b>Hjerte</b> i hele ti Aar Me trettes eller svettes . Thi hvorvel han fra sit 17 de til 27 de Aar , som han dengang ttnaede , af Satan blev angreben med utallig mange Pile og andre hactrde Slag , udholdt han dog alt dette , forblev standhaftig og uoverviudelig , uagtet han dog ikke havde Nogen om sig , der havde kunnet formane ham , eller forn var med ham i famme Trengsel ; ja , han havde Me engang noget Exempel herpaa for sine <b>Oine</b> , men de , som han daglig faa , vare ganste modfatte , faaat han ved dem lettelig havde kunnet fores bort fra den fande Gudsfrygts rette Lere ; derfor maa der fandelig have veret eu stor Kraft af den Hellig-Aand og guddommelige Gaver hos ham . Thi del havde jo veret lidt taaleligere for ham , hvis han havde havt endog kun en eneste Lidelsesfelle . Men alene , uden Formaning , uden Offer , uden hellige Predikener holdt han endnu stedse med stor Standhaftighet , fast ved den sande Gudsfrygt og et gudeligt Levnet , og blev under alt dette itte alene itte anderledes tilsinds , men han synes stedse at have tillaget mere og mere og vere bleven bestyrket i sin Standhaftighet». 74. Men nu cmgriber Djevelen ham ogsaa fra hoire Side , det er med Smigreri og Fristelse til Utugt og Vellyst , hvilken Anfegtelfe er lcmgl haardere og farligere , fornemlig for en Ingling , end alle de Storme og hctarde Slag , som komme fra Venstre . Thi omkring det attende Aar begynder Arvesynden at rore sig og rase ; da opstaaer der i Dnglingernes Hjerter forferdelige Lyster og Tanker , hvormed de fristes til Utugt . Joseph levede deugcmg omgivet af Hedningernes onde Exempler, hvorved Naturen , som af sig <b>selv</b> er lilboielig til Utugt , endnu mere kunde optendes og lottes , fornemlig hos et faadant sterkt , friskt Menneske . Thi saaledes siger Augnstinus etsteds : jo sundere vi ere af Legeme , desto storre og afsindigere bliver denne Syge ; den er Me saa heftig i et svagt Legeme eller i et , der er befveret med Sorger og Bekymringer, forn i et sterkt og sundt . 75. Saaledes levede nu denne Ingling i hele ti Aar i saadan Anfeglelse af den onde Lyst og kjempede mandig , idet han ikke eengang eller af een <b>Kvinde</b> alene , men ofte og af Mange blev fristet til utilladelig <b>Kjerlighed</b> . Thi unge <b>Kvinder</b> pleie ogsaa at fole denne Ulykke og Anfegtelfe , og nåar de bo famnten med unge Mandfolk , friste de dem paa mangehaande Vis fornt trette deres Hjerter fnart hid fnart did , om de kunde lotte dem lil <b>Kjerlighed</b> , fornemlig nåar de fe , at de ere stjonne nf Skikkelse og sterke af Legeme , faaat de uttge Mandfolk ofte have storre Moie med at verge fig mod saadanne Fristelser end med at modstact sine egne onde Lyster . Thi saaledes blive disse Ting nfmalede i Digterens Skrifter , forn hin Bonde siger hos Virgil : Ualo ms 6 alat6a polit , laßciva puslla , Nt luFit aci 83,1 i « 268 , & 86 cupit ants vicisri , d . e . : den kctade Galaten kaster et HEble paa mig ; derpaa flygler hun bag Piletreerne , men vil , nt jeg forst stal se , det er hende . Dertil kommer da ogsaa Djevelen med sin Kunst og List , hvormed han

<h4>3699</h4>ligeledes Matth . 24 , 13. : hvo forn bliver bestandig indtil Enden , han stal blive salig . Thi Djevelen bliver ikke tret , kan ikke hvile , men holder stedse ved , soger Aarsag og Anledning til at drive Folk til Synder . Derfor er det itte nok , blot een Dag eller eet Aar at verge sig mod ham eller kjempe med Moie og Arbeide ; det er fornodent, at man har Taalmodighed derhos og forbliver i saadan Strid med Standhaftighet , , saalenge indtil han enten seirer eller beseires og hans Hoved knuses . Men det er saare tungt for en Ingling i hele ti Aar at kjempe med en faadan Fiende , nemlig Kjodet og Djevelen , der stormer ind paa ham med faa mange Vaaben og en saa bekvem Anledning og Leilighed . Thi her forsomte han Intet , hvorved Joseph havde kunnet overvindes . Og dog forbliver han derunder standhaftig med et uovervindeligt <b>Hjerte</b> og holder ud iudtil Enden , ja saa , at han for denne sin store Kydsthed bragte sig <b>selv</b> i Fengsels og Lenkers haarde Kctctr . Dette er sandelig de Helliges rette Lidelse , ligesaa bitter som legemlig Pine uogensinde kan vere , og den havde vel fortjent at gjores stor og fremstilles i sit rette Lys af Andre , forn ere fynderlig veltalende . 89. Denne <b>Kvinde</b> tiltalte Inglingen saare uforskammet , idet hun siger : lig hos mig . Dog troer jeg ikke , at hun strax begyndte med disse Ord , foråt lotte og friste ham til Utugt ; hun beflittede sig vel paa , forst at iudtage ham med synderlig List og smigrende Tale , ctnstillede sig , som om hun var ham synderlig vel bevangen og elstede ham i Tugt og 2 - Ere , som det sig sommer for en from og det paa Grund af hans store Dyder og Dygtighed . Undertiden har hun vel ogsaa smilet til ham og talt venlig med ham , spurgt ham , hvorledes det stod til i Huset , hvorledes de andre Tjenere opforte sig i sin Gjerning , om de passede den flittig ? Undertiden har hun vel ogsaa givet ham Gaver for derved at vise , at hans tro Tjeneste behagede hende og hendes Mand vel . Hun for itte strax frem med disse Ord : lig hos mig , men angreb hans <b>Hjerte</b> med Smiger , ved Samtaler og mangehaande lonlig Lift , og banede sig saaledes Adgang til at rykke ud med denne Ovfordring til ham . 81. Og Joseph , der var sagtmodig og erbar af Naturen , fvarede hende venlig . Thi det pasfede sig Me for Tjeneren nt komme frem med stolte Ord mod sin Herres Frue . Derfor , da hun faa , at han undertiden ogfaa smilede venlig dertil og talte med hende i al Idmyghed, fattede hun derved stedse storre Haab i sit <b>Hjerte</b> , indtil hun mente , at han nu vilde vere overvunden ved hendes Godhed og venlige Samtale, dertil ogsaa ved den Velgjerning , som hun havde vist ham , og at hun nu uden Tvivl skulde faa Bugt med ham . Nu er det Tid , tenkte hun , og foer frem med disse Ord : lig hos mig . Thi hun mener , at han nu er ganste indtagen og overvunden ved saamange Smigerord , Fagter og Miner , som <b>Kvinderne</b> vel forståa at bruge . Dette synes ogsaa Moses at have villet antyde ved nt sige , at hun kastede sine <b>Oine</b> til Joseph , det er , hun vendte sine <b>Oine</b> paa ynglingen, saa stedse paa ham , beundrede hans Skjonhed og Dyd , priste ham ofte for de stjonne , erbare Seder , han havde , og ogfaa for hans Ustyldighed ; priste dertil ogsaa hans store Dygtighed , og banede saaledes efterhaanden Veien for den Skjendsel , som han stulde begact med hende . Men tilslut , da hun nu Me lengere bluedes ved at aabenbare den <b>Kjerlighed</b> , som hun lenge havde holdt hemmelig , farer hun frem

<h4>4715</h4>131. Disse Ord ere vift gctaede ham ner til <b>Hjerte</b> . Thi han beveges itte alene derved , at han ser sin Broder , men ogsaa ved at hore sin <b>Moder</b> , sin Fader og Alt , som var steet hjemme , omtale . Da lom han til at tenke paa , at hans milde , kjere <b>Moder</b> ynkelig dode ved denne Sous Fodsel , Aaret for end han blev folgt til Egypten . Da kom han vel ogsaa til at tenke paa , hvor megen og stor Ulytte hans Foreldre og ogsaa han <b>selv</b> havde maattet rtdstaa hele Livet igjennem , men fornemlig i de lo Aar , i hvilke Moderen dode og han solgtcs til Ismaeliterne , ved Enden af det famme Aar , hvori Benjamin fodtes . 132. Der er ingen Tvivl om , at dette faldt ham ind og traadte ham for <b>Oie</b> , da hatt faa sitt Broder , og hans <b>Hjerte</b> optendtes i ham af om <b>Kjerlighed</b> og hjertelig Godhed for Benjamin og hans Foreldre . Derfor siger hau : Gud vere dig naadig , miv Son ! Som om hatt vilde sige : al , med hvilke store Smerter fodte itte din kjere <b>Moder</b> dig ! Hvor mange store Besuerligheder , Bekymringer og have dog min Fader og jeg maattet bere , forend det kom saavidt med ung , at jeg atter fik fe Eder igjen og lunde vere fammen med Eder ! Ak " Gud , hvor furt det blev for ditt <b>Moder</b> , og for mig og din Fader , foreud jeg atter fik fe dig igjen ! Og uu strommer hans <b>Hjerte</b> ganske over .

<h4>6067</h4>Sidste , men de ere blevne de Forste , og vi , som vare de Forste , ere blevne de Sidste . Thi de hang saa fast ved Forjettelserne om den Schiloh , som stulde komme , at vi i Sammenligning med dem ere ganste kolde , sovnige og dorske ved denne vor Messias ' s rige og herlige Aabenbarelse og Komme . 407. Var det itte Synd , kunde mau nok fristes til at tilfoie sig <b>selv</b> Ondt , fordi vi saa lidet agte paa de guddommelige Gjerningers store Majestet . Thi er itte det en stor Gave og Herlighed , at endog en <b>Kvinde</b> i Nodsfald kan dobe og sige : jeg forloser dig fra Doden , Djevelen , Synden og al Ulykke og giver dig det evige Liv ; jeg gjor et Djevelens Barn til et Guds Varn . Men denne Aandens Rigdom ex i vore <b>Oine</b> bleven ringe , saa vi ikke agte synderlig paa den , fordi det ster daglig ; og dog er det visselig <b>sandt</b> . Evangeliets Tjener , som prediker og dober , er en storre Prophet end Jakob eller Moses ; vor Propheties Herlighed overgaaer langt hines . Vistnok havde de samme aandelige Spise , Dritte og Tro ; men vore Gaver ere saa rige , at endog et Barn kuude absolvere og briuge Folk fra Djevelens til Guds Nige og det itte ved noget Andet end Ordet , forn Efaias siger Kap . 11 , 6 , 8. : en liden Dreng stal drive en ung Love og fedt Kveg tilhobe , og et diende Barn stal forlyste sig ved en Dgles Hule og et afvant Barn stikte sin Haaud i en Vasilists Hule . 408. Derfor siger ogsaa St . Petrus 2 Brev . 1 , 19. : vi have det prophetiste Ord mere stadfestet , og I gjore vel , nanr I give Agt derpaa . O Kjere , mener han , giver Agt derpaa , holder i Troen fast derved , thi det er lutter <b>Sandhed</b> . Vi have den hellige Daab , Forlosning fra Doden og Synden . Thi Guds Son siger <b>selv</b> Joh . 3 , 16. : saa elstede Gud Verden , at han gav sin Son den Ecnbaarne , paadet hver den , som troer paa ham , itte stal fortabes , men have det evige Liv . 409. Paa denne Maade skulde vi nu sammenligne vor Herlighed og Rigdom med Fedrenes Aabenbaring i det gamle Testamente , paadet vi maatte ovvekkes og opfriskes ved dette store Lys og saaledes med glad <b>Hjerte</b> annamme og holde fast ved Evangeliet . 410. Men det er en skammelig Ting , ja at ynkes og grede over , at vi ere saa dovne og saa besatte af Sovesygen , at vi agte saalidet paa faadanne store Gaver og ere saa sovnige mod dem . Den stjendige , fordomte Synd , der lettelig befncerer os , forn der staaer Ebr . 12 , 1. , tillader os ikke at forlade os paa Gud , love og tatte ham med saadan Flid , Varme og Undren , som Sagens Storhed i sig <b>selv</b> krever . Thi hvis En troede , var det intet Under , om han dode af Glede .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100603016'target=_>Hofacker, Ludwig, 1857, Prædikener paa alle Søn- og Helligdage</a><h4>105</h4>Haand tilstikkct mig Meget . Ester min kjcere Faders Dsd vilde Stuttgarts Indvaanere have nug til Diakonus ved St . Leonhard og bestormede Kongen med Bonner og Underskrifter . Imidlertid faldt jeg ien lemfeldig Sygdom , der lidt efter lidt gik over til min tidligere Nervesvaghcd, saa at jeg fra Februar 1825 til Februar d . A . ei kan erindre mig at have lcest en Bog , foruden det guddommelige Ord og nogle korte Brevc . Om noget Arbeide i Herrens Viingaard var aldeles ingen Tale ; jeg var af Frelseren atter sat i Pensionsstand . Jeg henfaldt til Bekymringer ogsaa for mit fremtidige Livsovhold ; uden Formue , uden Indtcegter , uden Embede , priisgiven for en Sygdom , som efter min tidligere Erfaring kunde fcengste mig idetmindste i to Aar , med min <b>Moder</b> og min aandssvage Broder hos mig , som for en Deel ventede det Nsdvendige af mig . Desuden stulde jeg atter benytte et Bad og bruge andre Kure . Mine Udsigter vare ikke venlige ; men Herren ssrgede for os . Han brugte mine Stuttgartffe Venner til sine Redstaber , og jeg vil tåge min Tak til disse Folk med mig i Evigheden . Med megen <b>Kjcerlighed</b> og Fiinhed blev der ad de forskjelligste Veie og fra übekjendte Venner stedse sendt os saa Meget , at vi kunde leve uden Bekymring og desuden bruge de nsdvendige Kure . I August 1825 kom jeg tilbage fra St . Moritz og ventede nu i Stuttgart paa en Anscettelse , blev syg , kunde ikke lcese , ikke studere , haabede og atter haabede , hvor der Intet var at haabe , satte mit Haab til Herren under megen Vantroe , Tvivl og andre onde Tanker . Saaledes vedvarede det til Slutningen af October , da det behagede Gud at hjemssge mig med den haardeste Sygdom , som jeg nogensinde havde oplevet . Om November veed jeg aldeles Intet uden usammenhengende Drsmme og Phantasier . Det var den hsieste Grad af Nervefeber , og Lcegerne havde ganske opgivet mig . Men min trofaste Jesus , der ei vilde lade mig gaae fortabt og stedse soger min Sjcel med uudsigelig Taalmodigyed , besriede mig fra denne store Lidelse. Hvad de kjcerlige Brsdre i Stuttgart i . denne Tid have gjort mod mig , hvorledes de villlgen have holdt mig , vaaget over mig , lagt lisomslag paa mig , baaret og lagt mig , og hvorledes de ellers have viist deres Kjccrlighed , kan jeg ingensinde " glemme . Frelseren , der gjengjoelder en Draabe Vand , som man giver en tsrstig Discipel , han gjengjoelde ogsaa disse Brsdre , hvad de have gjort mod mig . Efter forgjceves at have meldt mig til Stammheim ved Ludwigsburg , ssgte jeg nu om Rielingshausen og blev ogsaa udncevnt dertil . Jeg snskede at prcedike endnu engang i Stuttgart; men da formaaende Folk fraraadede mig dette , saa afstod jeg derfra . Hvad jeg i denne Tid har erfartt mn . Guds trofaste og besynderligste Vaagen over hans Born , hvad jeg har erfarer om Dens Veie , der snart kommer med Glcede , snart med Smerte , dette ( hvoraf jeg her ligesom har leveret Eder en Ramme ) dette stal , som jeg haaber og beder , til evig Tid vcere mig i Minde " . " Noermere kan man vel ikke vcere Evigheden , " tilfsier han i en anden Skrivelse fra hun Tid , " end jeg var den under min Nervefeber . Men hvor bliver min Tak af , fordi Herren lod mig , ufrugtbare Trce , end lcengere staae ? Ak , min Frelser ! hvor bliver min Tak af ? Skal da mit haarde <b>Hjerte</b> ikke ved Glcrder , ikke ved Lidelsernes Ild , ikke ved Nogetsomyelst kunne bringes til Eftertanke ? Jeg har gjort den Erfaring , at Intet er tilstrcekkeligt , for at bringe et Menneske endog blot til en eneste <b>sand</b> Fslelse for sig <b>selv</b> og sin Gud , uden Christi Blod . Christi , Guds Lams Blod , maa til — og , o evige , korsfcestede <b>Kjcerlighed</b> ! — saa haard er jeg , og saa blod og naadig er du , at , da du saae , at ingen Magt var i Stand til at belive min dsde Villie , at der intet Middel fandtes , uden

<h4>120</h4>ogsaa maatte fuldkonnncs paa Esaus Huus , der for otte Aar siden blev omdannet til et aandeligt lacobshuus . Dette er et Anliggende , som ofte ligger tungt paa mit <b>Hjerte</b> , og som ofte driver mig frem for Naadens Throne navnlig for Eder Tubingcr . Tåger mig det ikke ilde op , jeg anmasser mig heived siet Intet ; men siden vor Moders Dod har Gud gjort mig mere prestelig for min Familie , end jeg forhen var . Var det nok , at give Guds Ord Bifald , saa kunde jeg allerede vcere mere tilfreds ; men Lidclsernes og <b>Kjcerlighedens</b> Mand vil have os ganske , og det beder jeg om baade for mig og for Eder . Jeg har i Einde , om Gud skjcenker mig Naade , at vore alt mere i Christo , at erklcere og opssre mig stedse mere bestemt mod Verden end hidtil . I Tubingen var jeg meget bestemt , men det var en Lovens Bestemthed ; senere blev jeg mere eftergivende , og nu vilde jeg gjerne blive et Menneske , < om vender sst Ansigt lige mod Jerusalem, en Afskye for den fromme eller ugudelige Verden . — Lever vel , kjcerc Brodre ! Gud oplyse Eders Hjerter med sin hellige Aaiid , og styrke Eder til al benytte Naadens Vid ; thi Verden forgaaer . " Ligcsom i hiint Brev , saaledes yttrede den Forev ! gede ogsaa ved andre Leiliccheder en svag Forudfolelse as , at hans jordiste Dagvccrk ei ffulde vare alt for lcenge . Dette saavelsom ogsaa den stille , hjertelige Lcengsel ester hans uforglemmelige <b>Moder</b> var blandt Bcvceggrundene til , at den Beslutning, som han aUercde lcenge havde omgaaets med , just nu kom til Modenhed, nemlig at lcegge Haand paa Udgivelsen af sine Prcedikener . Han udtaler sig herom <b>selv</b> saalcdcs : " Men hvorfor atter nye ' Prcedikener , da man af gamle allerede har alt for mange . Saa , vil v ? l Mangen spsrge . Jeg har <b>selv</b> spurgt mig saaledes , og dog fattet den . Beslutning , i Guds Navn at sende nogle af mine ringe Arbeider ud i Verden . Til denne Beslutning kom jeg paa fslgende Maade : " " F Stuttgart , hvor jeg prcedikede Evangeliet som VikariuZ fra Aaret 1823 til 182. ) , blev jeg siere Gange ovfordret til at udgive mine Prcedikc : ner i Trykken . Men jeg afslog det hver Gang med Bestemthed , fordi jeg ansaae mig og mit Arbeide for ungt og umodent , og derhos frygtedc jeg , at mit <b>Hjerte</b> skulde besmittes med Forfcengelighed . I Aaret 18 ) 6 blev jeg forflyttet hertil som Prcest . Min <b>Moder</b> reiste med mig og styrede , da jeg er ugift , min Huusholdning . Avo som kjendte hende , vil give mig Net i , at hendes Bortgang efterlod et foleligt Savn i mit Huus . Hun havde elsket mig saa smt , og vi vare ved stcerkere end blot naturlige Baand forenede med hinanden . Efter hendes Dsd fandt jeg det übeskriveligt tomt i mit Huus . Jeg tcenkte nu paa at beskjceftige mig med Noget forudcn mit egentlige Embede , som kunde give mit saarede <b>Hjerte</b> Ncering og tillige vcere til Gavn for Guds Rige . Til egentlige lcerde Arbeider har jeg hverken Evne eller Lyst . Saa faldt jeg paa den Tanke , at forberede nogle af mine her holdte Proedikencr til Trykken , og hidtil har jeg ei angret denne Beslutning . " " Men ogsaa andre Grunde bestemte mig hertil . Naar jeg hidtil vilde lcese noget i Scmdhed Grundigt , Clmsteligt , Opbyggeligt om Christo , saa maatte jeg ncesten bestandig tye til Skrifter , der vare udkomne fsr den sidste Halvdeel af forrige Aarhundrede . Jeg veed , at det er gaaet Mange som mig . Hvad han , der var en Lsgner fra Begyndelsen , i den nyere Tid har anrettet ved Syndens og navnlig Hovmodets Bedrag , hvorledes han under Lysets og Viisdommens Navn har fordreiet og forvendt de simple evangeliske <b>Sandheder</b> , hvorledes det er lykkets ham , at stille Jesu Christi Erkjendelse i Skyggen eller at forfalske den , og hvorledes han er

<h4>1313</h4>Det Fsrste , som vi i denne Henseende ville bemerke , er dette : den cananeiske <b>Qvinde</b> horte om Jesu og lod Ordet om ham gaae sig dybt til <b>Hjerte</b> . Dette kunne vi ste deraf , at hun bad Frelseren om Hjelp i Anledning af sin Datters sataniske Sygdom . Dersom Efterretningen om Frelserens Gjerninger ikke var naaet hen til ^ hende , saa havde hun jo aldrig gjort dette . Men vort Evangelium giver os endnu tydeligere Vink . Hun maa ikke blot leselig have hort om ham , men ogsaa forespurgt sig notere om ham . Dette er klart deraf , at hun raabte efter ham : " Herre , Davids Sem " . Hun var en hedensk <b>Qvinde</b> og vidste altsaa af sig <b>selv</b> Intet om den Forjettelse , som var givet David , og dog kalder hun Frelseren Davids Son . Hvorledes er hun vel kommen til denne Kundffab ? Ikke anderledes , end ved nsiagtige Oplysninger , som hun indhentede ; thi saadanne noiagtigere Bestemmelser blive sjelden meddeelte ved de fsrste Fortellinger . Naar Efterretningen om den nye jsdiske Prophet kom hen til de tilgrcendsende hedenske Lande , saa indcholdt den vel ikke Andet , end at der var ovstaaet en Undergjerningsmand , at denne var fra Nazareth og havde vct > sit Ord allerede helbredet mange Syge , uddrevet mange Djevle , forvandlet Vand til Viin i Cana o . s . v. Lengere end til saadanne udvortes Begivenyeder strekke ikke gjerne de Fortellinger sig , som gaae igjennem Folkets Munde . Men dermed kunde den cananeiske <b>Qvinde</b> ikke lade sig nsie ; hun forskede videre ; hun horte , at han nedstammede fra Davids Slegt og maaskee var den forjettede Davids Son ; hun forespurgte sig om hans Handlemaade , om hans Hjertelag , og horte om hans Kjerligyed til de Ulykkelige , om hans store Barmhjertighet ) . Og det , som hun saaledes hsrte om ham , det festede Rod hos hende og frembragte en übegrendfet Hsiagtelse for Jesu og en urokkelig Tillid til ham . O mine kjereTilhsrere ! Hvad skulle vi sige om denne hedenske <b>Qvinde</b> , der annammede og benyttede de faa og utilforladelige Efterretninger, hun fik om Jesu , saaledes , at af disse smaa Fro hendes sterke Tro , som Frelseren <b>selv</b> beundrede , kunde fremvoxe ? Hvad stulle vi sige om denne <b>Kjerlighed</b> til <b>Sandheden</b> , om denne Troskab iat bevare det Modtagne ? Sandelig , denne Grund er fremstillet som et lysende og fordummende Erempel for os og denne Tids Mennesker. Dog ikke alene denne <b>Qvinde</b> ; men Gud har i vore Dage

<h4>2261</h4>fremsig ^ r et Sprog udenad eller lader sig nsie med blot at lese og give sit Bifald ; mon dm Hellig ' Aand og hans Lys behsve vi , dersom denne Hemmelighed om den korsfestede Frelser ffal blive os en Guds Kraft . Ak derved bedrage Mange sig . De see vel ogsaa op til lesum , troe paa den Korsfestede , men ikke med aandelige Sine , ikke med et saadant <b>Hjerte</b> , som den Hellig Aand kan oplyse . De ville det ikke ; de mene at kunne hjelpe sig <b>selv</b> , gjere sig <b>selv</b> vise , og saa forblive de dsde i sine Synder . Men hvo som seer op til Frelseren med Sine , som den Hellig Aand skjcrnker , han kan ikke mere forblive dsd ; hvo som i Aand og <b>Sandhed</b> erfarer : den korsfestede <b>Kjerlighed</b> er min ! han har et evigt Liv ; og dertil er ingen naturlig Kraft tilstrekkelig , det er Guds Gave . — Men kjere Tilhsrere! hvor stor er dog Guds <b>Kjerlighed</b> til os , at han har givet os saa let et Middel til at blive salige ! For os er ikke en Slange fremstillet , men Jesus , den Ophoiede ; hvo som i Troen seer op til ham , han stal finde Naade . Dette Middel passer ikke for en crrbar , dydig Mand , som har nok i sig <b>selv</b> ; ikke for en dydig , sedelig <b>Kvinde</b> , som mener at komme i Himmelen , fordi hun opfsrer sig anstendigt; men det passer for Fattige i Aanden , for Syndere , som ikke kunne hjcelpe sig <b>selv</b> , hvis egen Mre , egen Kraft , egen Dyd er forsvunden , som ikke mere finde nogen Trost i sig <b>selv</b> og torste ester Naade , efter Barmhjertighed . For Eaadanne passer det , for dem alle uden Forskjel , hvao enten de ere Christne , loder eller Hedninger. Alle de , som troe paa den Korsfestede , stulle ikke fortabes , men have det evige Liv .

<h4>2328</h4>fr kommen til Verden , ikke for at dsmme dia , men for at frelse dig , for at frelse dig Ulykkelige , Usle , Fordcervede , som havde fortjent Hel : vede ! — O sode Evangelium ! hvo kan noksom prise eller udtale del ! hvo kan udsige Herrens store Gjerninger ! Frelse vil han det Fortabtl , Men det er ogsaa nsdvendigt , at dette prcrdikes og bevidnes de » ene Gang ester den anden , for - at Kraften af denne <b>Kjcrrlighed</b> kai , overvinde vort onde , vantroende <b>Hjerte</b> ; thi af Naturen kunne vi ikl , troe det . Vi kunne vel troe Meget af os <b>selv</b> ; vi kunne troe , a , Gud er almcrgtig , at han har skadt Verden , at han endnu fremdele er Verdens Herre og styrer den efter sit Raad ; vi kunne troe , a > der forestaaer os et Regnstab , og at han har sat en Dag , paa hvilken han vil demme Levende og Dsde ; vi kunne troe , at der er « Liv efttr Dsden , en Himmel og et Helvede ; alt dette kunne vi t > > Ned af os <b>selv</b> troe og fatte ; — men at Menneskens Sem er kM men for at ssge og frelse det Forlabte , at Gud har saa store Fredb tanker over os , at han har givet os sin eenbaarne Son , paa det al vi stulle leve ved ham — det kunne vi af egen Kraft ikke fatte saaledes, som det maa fattes ; dertil behsve vi nsdvendigviis den Helliz Aands Kraft ; thi til at troe dette ere vi for svage , for blinde , foi udygtige , altfor fordcervede , for mistroiske og fiendffsindede mod voi Gud . O det er et stort Under , naar denne salige og saliggjorenbi Tro opkommer iet Menneske . I , som have erfarct det , I ville sell tilstaae , hvilken Gudskraft der udfordres til , at Mennesket kommi ud af sine morke , mistroiske Tanker om Gud og lcerer at troe Guds <b>Kjcerlighed</b> over ham , hans forbarmende <b>Kjcerlighed</b> , hans hulde , venlige <b>Hjerte</b> , hans tilgivende Naade , hans mere end <b>moderlig</b> opofrenei Trofasthed over ham . Sandclig , det er et stort , et helligt Under , Vist nok , saalcenge Mennestet er dsdt i sine Synder og vedbliver si » Syndesovn og tragter efter det , som er paa Jorden , saalcenge ha » endnu ikke fornemmer og foler Guds Retfcerdighed , men speiler sig > sin eqen Retfcerdighed , og i al sin Usselhed indbilder sig at van from ! saalcenge er det ham enten en ligegyldig eller endog modbydelig Tale , at Menneskens Sem har et saadant forbarmende , grundgod ! Hjertelag mod Syndere , mod fortabte Syndere ; eller han mener maaskee, at han troer det , men troer det dog ikke . Men ganske anderledes er det , naar Mennestet engang begynder at strcrbe efter en fullkommen og <b>sand</b> Forening med sin Frelser , naar han vil optage de ! dyrebare Ord om den fuldbragte Forlssning og om Guds naadigl Hensigter med ham i sit <b>Hjerte</b> , og hungrer og tsrster efter dette Retfcerdighedens Ord og dets Kraft . Naar det da hedder : " jeg vil staae op af mine Synder og gaae til min Fader " — ak , da mcrrker man , at man ikke kan troe det ; da stjemner og erfarer man , at vi ikke kunne stue ind i Guds <b>Hjerte</b> saaledes , som vi gjerne emstede ; da bliver det saaledes , at man med Toldercn ikke vover at oplsfte sine Sine ; da mener man , at Guds Son er kommen for at domme

<h4>3125</h4>Man kommer paa Troens Vei undertiden i Tilstande , i hvilke man er ncer ved at udraabe : har da Herren forglemt mig ? har da hans Barmhjertighet ) en Ende ? vil han da ikke igjen opflamme den lille Gnist af Tro , som er ncer ved at udslukkes i min Sjcel ? O hvor sod er i saadanne Timer den Tanke , at vi ere hsit agtede i hans <b>Oine</b> , og at dette hans Sindelag mod os ikke forandres ! hvilken Trost flyder ned i det bekymrede <b>Hjerte</b> , naar det tsr tro : Herren, min Gud , er stsrre end jeg ; han elsker mig mere , end jeg elsker mig <b>selv</b> ; jeg er hsiere agtet af ham , end af mig <b>selv</b> ; hans usigelige Forbarmelse overstiger mine Tanker og vil ikke , at jeg skal gaae fortabt; om jeg ogsaa vilde opgive mig <b>selv</b> , saa vil han dog ikke opgive mig ; thi hans Skaberkjcerlighed omfatter mig , hans dyrebare Blod er udsst ogsaa for mig ; han kan ikke glemme , hvad jeg har kostet ham , hvor dyrt jeg er kjsbt . — Ja i <b>Sandhed</b> , kjcere Tilhsrere ! saadanne Tanker kunne jo vel igjen styrke og opreise et semderknuset <b>Hjerte</b> , og jo dybere Sjcelen erkjender sin Uvcerdighed og Fordsmmelighed, desto mere ere de istand til at knytte Hjertet med uoplsselige <b>Kjcerlighedens</b> og Taknemmelighedens Baand til Frelseren , i hvis Barmhjertighed det tsr nedscenke sig ligesom i et Hav , og for hvis Skyld det uagtet al sin Elendighet ) dog er dyrebart i hans Faders <b>Oine</b> . Men ogsaa vore Brsdres og vore Ssstres Ejcele ere hoit agtede i Herrens Sine , og vi ffulle ogsaa agte dem hsit . Seer til , siger han , at I ikke foragte Nogen af disse Smaa , og forarger ikke

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112200023'target=_>Hofacker, Ludwig, 1857, Prædikener paa alle Søn- og Helligdage tilligemed andre Taler</a><h4>49</h4>igjen , hvad jeg har gjort stet . Dog ofte , naar jeg er fattigst , naar ethvert klart <b>Blik</b> md i G ) igheden og paa Gud er mig bersvet , og jeg beder Herren , at han i min Armod vil give mig Noget for de Syge , da gaaer det ofte ret godt , ofte langt bedre , end om jeg gik frem med halv Tillid til egen Kraft og halv Tillid til Naaden . Naar jeg saaledes vandrer fra det ene Huus til det andet , pleier jeg at anraabe Herren :  Giv mig igjen Noget ! — Dog , anste mig ikke for from , kjcereßrsdrel det er jeg aldeles ikke , men jeg er , det er <b>Sandhed</b> hvad jeg skriver , en ugudelig , utro Skabning . Der mangler mig saa meget af den sande Vennens Aand , og saa Icenge jrg ikke har denne , kan det heller ikke blive bedre med mig .  leg beder Eder , kjcer ? Brsdrel at I i denne Henseende anraabe Herren for mig , iscrr om at han maa gjere mia ^ mere tro , paa hvilkensomhelst Maade ; thi Vennens Aand er iscer bunden dertil , at man med Trostab bevarer hans Naade ; han forestriver ogsaa Betingelser for denne Naade . En Sjceleforger har tunge Sorger i sit <b>Hjerte</b> , nemlig de Sjcele , som Herren har betroet ham , at han stal vogte dem . Men hvad nytter al egenmcegtig Vogten ? At bede for sin Menighed er en Hovedsag . Da ere vi fsrst sande Guds Prcester , naar vi med vor Ben trcede frem for Gud for Sjcelene — paa en vis Maade nemlig — thi den , der egentligft stal trcede frem for dem , er Christus , den eneste og evige Ippersteprcest . Uden en ivrig Bsn for Menigheden vil der heller ikke 1 Sjcelesorgerens <b>Hjerte</b> kunne findes nogen <b>Kjcerlighed</b> til hans Sjcele , og uden <b>Kjcerlighed</b> kan Mennesket intet <b>Sandt</b> « drette .  Med den christelige og broderlige Forden , kjcere Brodre ! jeg tilstaaer det for Eder , gaaer det mig ofte slet . Jeg drives stedse til Forden , og dog er hver Gang min egen Elendighed saa stor , at jeg ikke med fortrestningsfuldt <b>Hjerte</b> beder for Andre , men snarere tcrnker : Hvad bevceger Dig til at bede for Andre , da Du <b>selv</b> fattes Alt ? Vil vel Gud hore paa vin fyndige Bon for Andre ? Thi i Forbsnnen harve vi os til et prcesteligt Standpunkt . Men nu maa en Prcest have indre Vcerdighcd og Kald dertil , og dette fattes mig . Og saa falder Bonnen mig overhoved ogsaa suur , og derfor iscer Forbonnen , fordi jeg endnu ikke elsker min 3 ^ ceste som nug <b>selv</b> ; og dette er da en elendig , kummerlig Trceldomstilftand. Dog , Seimen vil gjere mig fri , hvis jeg ikke ganske unddrager mig hans Belcerelse . Jeg troer , at man maa gjennemgaae flige Tilstande , og tsr ikke , ja kan ikke slippe fra dem , fsrend Herren gjor os ftie .  Min Menighed er , som jeg allerede har skrevet , tilfreds med mig , og jeg troer , at de Prcedikener , feg allerede har holdt , ikke have vcrret uden en blivende Velsignelse . Dog , Herren alene kjender Hjerterne . Vi udsaae og vente , indtil Frugten modnes ; men Avindsmanden er ogsaa virksom, ifcer i ncervcerende Tid . Jeg foler tydeligt , at det nu er en Indsamlingens Tid . Der er udgangen en Naadersst fra den Levende , som siger : Kommer til mig , IMennesteborn ! Thi , hvor er nogensinde sundet en saadan Hunger eftcr Guds Ord , som i vor mcrrkvcerdige og afgjsrende Tid ? Naar var der en faadan Iver ester at hore <b>Sandheden</b> , jeg siger ikke blandt Guds Bern , men blandi Verdens Born ? Naar fandtes saadanne Aandens Dragelser ? I kunne mcerke paa Eders eget <b>Hjerte</b> , hvad Herren gjor i denne Tid . Giver Agt ! Der udsses Krafter fra oven . Lader os trolig benytte deune Tid , sacwel til vor egen som til Andres Grundfckstelse og Befceftelse i Troen og Haabet , paa det vi maae styrke os til Fristelsens Stund l Jeg lceser nu , naar jeg har Tid , Bengels Forklaring over lohannis Aabenbaring . Den ovenfor udtalte Dom om vor Tid er imidlertid ikke fremgaaet af dette Bengel ' ste System , men af lagttagelse

<h4>81</h4>give Ham Wren ; men de ere blinde , omtaagede af Tidens Aand , af denne Verdens Gud . O Brsdrel beder , iler hen til Guds Lam ! Bliver for Guds Skyld Born , saaledes som Frelseren har befalet , troer paa hans Ord , foragter Verden med hele dens Viisdom , og strides ikke ! — Mene I , at Levi Vern , naar den store Smelter engang kaster dem i sin Digel ( Mal . 3 , 2. 3 ) , endnu ville strides og komme med sine Meningers Kram . Man lider i vor Tid netop af en overmaade stor Kjedsomhed og Vellevnet . Seer Eder for , man vil forbittre Eders kjedelige Rolighed ; Herren vil gjsre en Ende paa al unyttig Snak , derom er jeg overbevist . "  leg gjor den Erfaring , " tilfoier han i et andet Brev ,  at netop denne raadne Plet tilhsrer vor Tid ; man hender ikke loenger sig <b>selv</b> eller sin fuldkomne Afhcengighed af Gud , og feler ingen Skrcek for ham og hans Retfcerdighed ; man frygter ham ikke lcenger , men de Herrer Philosopher have med sine elendige Gjoglerier tilsmurt vore Dine , saa at den levende Guds Majestcetsrettigheder ikke lcenger blive anerkjendt ^ af denne elendige Sloegt . Man veed det ikke lcenger , og bencegter ' det , og det falder ikke engang den ncervcerende Verden md i Tankerne , at Herren , vor Gud , er en fortoerende Ild , der kan fordcerve Legeme og Sjoel og som , hvis vi ikke tåge vor Tilfiugt til den aabenbarede <b>Kjcerlighed</b> , vil fordcerve dem i Helvede , hvor der er Graad og Tcenders Gnidsel . — Ere dette ikke selve den evige <b>Sandheds</b> Ord ! Men vor oprorte og dog saa kraftlsse Slcegt lader det ikke mere gjcelde , hvad den evige <b>Kjcerlighed</b> og <b>Sandhed</b> har udtalt med saa stort guddommeligt Alvor , men Christus bliver draget ned i denne Tids Ligegyldigheds og Lunkenheds Dynd , og de sige da :  10 , nu have vi vor Gud ! " Men denne Tale vil ophsre , naar han kommer i Himmelens Skyer , og hvert <b>Die</b> skal ste ham , ogsaa de , som ham gjennemstunge ; og alle Jordens Sloegter skulle hyle for ham ( Aab . 1 , 7 ) . — O , det har dog saa inderligt rort mig , hvad jeg lceste om Dr. Martin Luther : hvilken " Skrcek han har haft for den yderste Dom , og vilde dog maaite blive salig . — Er det da nu blevet anderledes ? Har da Gud forladt sine evige Majestcetsrettigheder ? Ere da hans Domme blevne til Bsrneleg , hvortil denne Tids Menneskers letsindige og frcekke Aand gjerne vilde gjore dem ? — O , Brodre ! Herren fordrer et retskaffent <b>Hjerte</b> mod ham fra vor Side ; ellers vil han , naar han gjennemrenser sin Lo , ogsaa bortkaste os som Avner og opbrcende os med uudsiukkelig Ild . Ak , lader os dog bede ham om Diensalve ! " At det uscrdvanlige Bifald , som den Salige endnu som ung Prcedikant hsstede i Landets Hovedstad , maatte udscette ham for Faren af at rykkes bort fra den ydmyge Eenfoldighed i Christo , behsvcr vel ikke nogen sceregen Antydning . Men Gud havde allerede i sin Viisdom sorget for en jtlstrcekkelig Modvcegt i en stadig Sygelighed , ligesom ogsaa i mange Slags huuslige Lidelser ; og det manglede heller ikke paa den Helligaands trofaste Naadetugt . Hvor nsie den Forevigede pleiede at tåge det med sig <b>selv</b> i siige Ting og hvor strcengt han pleiede at vaage over sig <b>selv</b> , fremgaaer af nogle Åttringer , som ligeledes findes i ovenstaaende Brev . Han siger :  Medens det i mit Indre seer saa jammerligt ud , anseer man mig i Stuttgart for een af Christi frommeste og meest benaadede Disciple . Jeg har <b>selv</b> givet nogen Anledning til denne Mening . Ved visse Vendinger i Foredraget kan man give sig Udstende af at man virkelig lever i Noget , hvori man dog , naar <b>Sandheden</b> skal frem , isse lever . Om jeg nu end ikke med mit Vidende har lagt an paa et saadant Hykleri , saa har dog den Tanke : at Dette eller Hiint maatte siges , — den Bevidsthed , at jeg ogsaa allerede paa en vis Maade havde erfaret hos mig <b>selv</b> , hvad jeg maatte sige , om end ikke netop nu , da jeg siger det , — Synet og Tiltrakningen

<h4>119</h4>aabenbaret i mit <b>Hjerte</b> ; han har hidtil fort mig ved Haanden som den ommeste <b>Moder</b> . Jeg maatte skrive Folianter , dersom jegvilde fortcelle alle hans Naades Undere . Dette vcekkcr hos mig Begjcerlighed efter ogsaa at frembringe Noget , der kunde fremme hans Rige . Og om end dette Arbeide aldeles ingen Nytte skulde stifte , saa har jeg dog i det mindste haft den gode Villie og ikke skyet Jesu Christi Forsmcedelse . Ikke at jeg dermed vilde gjore noget Fortjenstligt — dette vcere langt fra ! — men jeg veed godt , hvormeget Urcent der ogsaa flyder med blandt vore bedste Gjerninger ; jeg haaber heller ikke paa Andet end paa Barmhjertighed — : men , hvem vil forarges paa mig , naar jeg , saa vidt det staaer til mig , soger at bcere en liden frem til Bygningen af Zion , enten den nu duer eller ikke . Om den duer , kan alene yen vise Bygmester rigtig bedsmme ! " Herren forlenede ham ogsaa Kraft rg Mod til endnu <b>selv</b> at udgive to Hefter med 22 Prcedikener . Dommene om dette Arbeide vare naturligviis i Vegyndelsen meget forskjellige . At Folket , som sogte Opbyggelse , med Glade greb efter det , vil man finde naturligt : men mange Dannede, navnlig ogsaa Theologer og Prcester , <b>selv</b> af det bibcltrocnde Parti , rystede paa Hovedet og kunde ikke tilgive den ungdommelige Evangelist , at han saa hensynslost havde aflagt den gamle , afmaalte , i Veviisform vptrcedende Prcedikcmaades Vaabenrustning , og ganske simpelt , men dristigt og muntert traadte frem med det guddommelige Ords Slynge . Nogle meente , at Farverne vare for stcerke i Skildringen af den menneskelige Grundfordcrrvelse ; Andre , at Sjcelelcrren ikke kom til sin fulde Ret for lutter Opfordring til Bod og Omvendelse , atter Andre , at det uforbeholdne Vidnesbyrd mod Vantrocns og Tidsaandens Vranglcere ikke havde sin Plads paa Prcedikestolen . En trofast Ven gjorde ham opmcerksom paa disse Bemcerkninger ; den Salige gav ham herom folgende Svar i et Brev fra Foraaret 1828 : „ Din Dom om disse Prcedikcner . lader jeg staae hen ; men angaaende det Andet , som Du tilskriver mig ' om en god Ven , kan jeg neppe tie stille . Jeg kan ikke anerkjende denne Dom , hvorledes jeg saa end betragter den . Jeg stal overbyde den hellige Skrift ? Dette er jeg mig ikke bevidst . Kan man overbyde Skriften , eller forstcerke dens Loerdomme? Man kan jo desuden ikke udtale det Bedste , som ligger hos En ; det er usigeligt . O , hvis man kalder min usle Stammen  at ville overbyde", saa veed jeg ikke , hvad jeg stal tcrnke . Det er <b>sandt</b> , derpaa arbeider jeg og sikkerlig Enhver , som efter Evne strceber at arbeide i Herrens Viingaard , nemlig paa at fremhceve Bibelens Loerdomme , saa godt jeg kan ; det har ofte vceret en haard Fodsel , fsrend jeg har sundet den Maade , paa hvilken jeg paa bedste Maade kan gjore en <b>Sandhed</b> tydelig og indtrcengende hos Mennesket . Jeg anvender ikke ringe Umage paa det . Men er det ikke min Pligt , at jeg ssger at komme til Hjertet , saa godt jeg kan ? Sove de ikke , hvis jeg ikke gjor det , hvilket jeg nok har erfaret ? Skal jeg tale saa kjedeligt som muligt om Frelseren ? O , jeg onskede , at jeg endnu mere kunde overbyde , hvis man vil kalde det saaledes ; jeg onsiede , at jeg kunde ajsre mine Ord til Spyd og Nagler for mine Tilhsrcres Forstand oa , Hjerter . Denne Dom kan jeg ikke sinde mig i .  Han siger videre :  , jeg opfatter Menneskenes Tilstand anderledes end Christus og Apostlene . " " Har han vel ogsaa loest Rom . 3 ? og Rom . 7 ? og kastet et <b>Blik</b> ind i sit stinkende <b>Hjerte</b> ? Ak ! man er i vore Dage kommet langt bort fra * Fcedrenes Tro og deres Eenfoldighed . - Jeg tanker ikke paa at tale , taler heller ikke og kan heller ikke tale saaledes som Arndt , Spener , Brastberger og vvre gamle Wurttembergere . Det kan man ikke

<h4>1039</h4>Christen , at leve i denne Verden og at leve for Evigheden . Denne Vildfarelse er temmelig almindelig blandt Menneskene . Hvad forstaae vel de Fleste ved <b>sand</b> Fromhed ? Forstaae de ikke derved en Sindstilstand , i hvilken man ster ikke mere tenker paa Andet end himmelske Ting , hvor man aldeles ikke mere bekymrer sig om det Jordiske , altsaa bliver nersten udygtig til at beskjeftige sig med Andet end aandelige Forretninger ? Vel have de lunkne og verdsligsindede Christne og de nyere Dplysningsmend , som lcere en munter Christendom , hvorom jeg har talt i Indledningen , gjort denne Forestilling temmeligt Afbrek , og ved deres Vandel og deres Ord bragt mange Mennesker til det Modfatte , nemlig til den Anskuelse, at <b>Kjcerligheden</b> til Verden og til det , som behager Kjedet , godt lader sig rime sammen med den sande Christendom , ja efter Guds Villie horer med til den . Men dog hcenger den Fordom endnu ved Mange , at et Menneskes jordiske og himmelske Kald ikke lader sig forene , og den maa opstaae hos alle dem , som vel have det rigtige Begreb om Storheden af det himmelste Kald , men som derhos ikke fsre de jordiske Forholde tilbage paa Gud , men kun betragte dem som noget Tilfeldigt , som en Tumleplads for vore Lidenskaber og vor Egenvillie . Af denne Fordom kom det , at allerede faa Aarhundreder efter Christi Fsdsel den Mening blev herskende i Kirken , at den , der vilde fsre et virkelig gudeligt Liv , kunde umuligt lengere indlade sig med jordiske Beskjeftigelser ; han maatte trekke sig ud af Verdens Larm og soge Eensomheden , for der at anvende hele sin Tid i Bon , Betragtninger over den hellige Skrift og andre Andagtssvelser. Paa den Maade har det undertiden hendt , at A3gtefeller, som ved den Helligaand vare blevne vakte til at soge Guds Rige , have forladt hverandre , Manden har begivet sig tilhsire i Srkenen , <b>Konen</b> tilvenstre i Orkenen , hvor de have neret sig af usel Fsde , og tilbragt hele deres Tid i Omgang med Gud i Bonnen og ofte ikke gjenseet hverandre alle deres Livs Dage . Det kaldte man at forlade Verden . Men den rette Maade at forlade Verden paa bestaaer ikke deri , at man drager ud i en Brken , hvorhen man ogsaa tåger sig <b>selv</b> med , men deri , at man daglig afdser fra sin Phariseer og sin Sadduceer , og sin falske Prophet , og sin Antichrist , og sit Verdensmennefke , som man gaaer med i sit <b>Hjerte</b> , og dette maa skee midt under Livets Tummel og Storme og blive fuldbragt , ikke ved at rende og lsbe hid og did , men ved Guds Naade og Forbarmelse . Troer dog ikke , kjere Tilhsrere , at der nu ikke lengere findes Folk , som gjerne gad sve sin Christendom i Orkenen , ligesom ide fsrste Aarhundreder . Hvor mangen Begynder i Christendommen sukker ikke endnu i sit Indre : Ak , om jeg kun havde

<h4>1060</h4>fuldfsrer endog det ringeste Arbeide med dette Sind , behageligere for Herren , end om hun paa den Tid , da hun skulde udfsre et saadant Arbeide , havde ligget paa sine Knee og bedet . — Og naar en <b>Huusmoder</b> har Born , og maa vadske disse Born , stikke paa deres Klaeder , afholde dem fra Uartigheder og hun samtidigt tillige maa koge og forestaae sin vanskelige Huusholdning , hvor undertiden det Ene stormer md efter det Andet : saa har hun vistnok i det daglige Liv mange Fristelser til at blive utaalmodig og til forskjellige Udbrud af ster Lune . Men , naar hun besinder sig , og fatter den faste Beslutning og siger i sit <b>Hjerte</b> : Du har anviist mig denne Stand , kjaere Frelser , og da disse Ting komme fra Dig , saa vil jeg nu for din Skyld forssge , om jeg ikke derved kan bevise og vedligeholde Troffab og Fatning og Taalmodighed ; giv mig kun Viisdom dertil og Kraft fra din Helligdom til dette moisommelige og urolige Liv ! Og hun gaaer hen og atter med frisk Iver i Jesu Navn tåger fat paa den Gjerning , hvortil Gud har bestemt hende ; se saa gjor hun en Gudstjeneste og er ligesaa from , som om hun var i Kirken og fornam de skjonneste Rsrelser i sit <b>Hjerte</b> . — Og naar en Haandvcerker sidder paa sit Vcrrksted ved sit Arbeide og tcenker hos sig <b>selv</b> i sit <b>Hjerte</b> :  Det er vistnok et noget kjedsommeligt og trettiende Arbeide , og desuden er Fortjenesten just ikke saa stor , men naar jeg for min ringe Dccl ved mit Haandvaerk kunde gjore Dig A3re , store Frelser, saa vilde jeg fsle mig overmaade lykkelig . Ak , giv mig dog fornsden Viisdom og Trostab , saa at jeg udfsrer mit Arbeide saa godt og nsiagtigt som muligt ; bevar mig ogsaa fra ved Uagrsomhed eller Egennytte at bedrage Nog / n for det Ringeste : " — jeg siger , naar en Haandvcrrksmand tcenker saaledes og altsaa for Jesu Skyld anvender den stsrst mulige Flid paa sit Arbeide , saa gjsr han af sit ydre Kald en <b>sand</b> Gudstjeneste og af sit Vcerksted en Kirke og en Guds Helligdom . — Og naar en Bonde fsrer sin Ore paa Torvet og han forud anbefaler sin Sag i Frelserens Haand , og det for Frelserens Skyld er ham om at gjore at han under Salget ikke maa lyve , ikke rose den for meget , ikke bruge unyttige Ord , men lade sin Tale vcere : ja , ja ! og nei , nei ! og han heller vil lide Skade end tilfsie sin Nceste Skade — Alt af Lydighed mod og <b>Kjcerlighed</b> til Frelseren — , saa har han gjort en Gudstjeneste af sin Handel . Behsver jeg endnu flere Exempler? Jeg haaber , at Inu godt forstaae , hvad jeg mener . Alt Arbeide , i ethvert Kald , hvilket Navn det saa bcerer , maa stee for Herren ; thi ikke det , som man gjsr , enten det er stort eller lidet , men det Sindelag , hvormed man gjsr det , gjsr alene Udstaget for ham . Se , paa den Maade kan man gjsre en Gudstjeneste af Alt , af Spisen og Drikken , af Ssvn og Waagen , af

<h4>1489</h4>Band til Biin o . f . v. Videre end til saadanne ydre Kjendsgjerninger naae ikke lettelig de Fortellinger , som gaae i Folkemunden. Men dermed kunde den kanancriste <b>Kvinde</b> ikke sinde sig tilfredsstillet ; hun forhorte sig videre ; hun horte , at han ncdstammede fra Davids Slcrgt , og maaste var den forjcrttede Davids Son ; hun sogte Underretning om hans Dpfsrsel , om hans Hjertelag, og horte om hans <b>Kjcrrlighed</b> til de Elendige , om hans store Barmhjertighet Og hvad hun saaledes horte om ham , det frembragte hos hende en übegraendset Hoiagtelse for Jesus og en urokkelig Tiltro til hans <b>Hjerte</b> . O , mine kjoere Tilhorere ! Hvad stulle vi sige om denne hedenske <b>Kvinde</b> , som faaledes aunammede og benyttede de ringe og upaalidelige Etterretninger , hun havde om Jesus , at den staerke Tro , som Frelseren <b>selv</b> beundrede hos hende , kunde vore frem af dette lille Frokorn ? Hvad stulle vi sige om denne <b>Kjcerlighed</b> til <b>Sandheden</b> , om denne Trostab i at bevare , hvad hun havde modtaget? Sandelig ! Denne <b>Kvinde</b> er sat som et lysende og fordsmmende Tegn for os og denne Tidsalders Mennesker . Mange Hedninger , som dog ikke kjende Guds Vei , vise for ncrrvcerende den stsrste Begjcrrlighed ester Guds Ord og ester Loerere . Folk , som forst for kort Tid siden have lcrrt Evangeliet at kjende , lcere med den stsrste Begjerlighed at loese , for at kunne lcese Guds dyrebare Ord ; de agte Guds Ord for sin dyrebareste Skat ; de kunne ikke blive mcrtte af at hsre om sin Jesus ; de soge Frelseren ; de soge hans Ansigt ; med mange Bonner og Taarer soge de Guds Naade og sine Synders Forladelse i Lammets Blod . Herrens Herlighed opgaaer over mange Hedninger , og vi ste dem vandre i dens Lys . 33 i derimod have Guds Ord i saa rigeligt Maal ; vi have det hjemme og i Kirken , og agte denne dyrebare Skat som noget Almindeligt , og det forekommer os , at det saa maa verre . <b>Sandheden</b> , det evige Evangelium have vi i saa rigeligt Maal , og vi rore os ikke , og de Fleste blive i sine Synder, i sit Liv ester Fcedrenes Viis , i sin aandelige Uforstand og Blindhed , og takke ikke for Guds « fortjente Forbarmelse , og tcrnke ikke paa at takke . Hvad vil dog Gud maatte foretage sig med os ! Naar han lader sit dyrebare Evangelium prcrdikes for os ; naar han lader Christum afmale for vore Vine ; naar han lader et saa rigeligt besat Bord fremscrtte for os , og vi ikke aabne Munden for at nyde , men for en Deel endog vcemmes ved denne Spise , medens mange Tusinder vilde vare glade ved de Brodfmuler, som falde fra vort Bord ; hvad vil da Gud blive nsdt til at foretage med sin dovne , kolde , mattte , stolte Christenhed ? Det vil han maatte foretage med den , at han aldeles omstoder dens Lysestage , hvortil der i vor Tid allerede er Udsigter , saa at

<h4>2514</h4>Mange sig ! De stue vel ogsaa op til lesum , og troe paa den Korsfestede , men ikke med aandelige <b>Oine</b> , ikke med et saadant <b>Hjerte</b> , som den Helligaand kan oplyse . De ville det ikke ; de troe , at de kunne hjelpe sig <b>selv</b> , kunne gjere sig <b>selv</b> visebog saa blive de dsde i sine Synder . Men den , der fluer op tilMelseren med Dine , som den Helligaand skjenker , han kan ikke lengere forblive dsd ; den , som i Aand og <b>Sandhed</b> gjor denne Erfaring: Den korsfestede <b>Kjerlighed</b> er min ! han har det evige Liv ; og dertil naaer ingen Naturkraft , det er Guds Gave . — Men , kjere Tilhsrere , hvor stor er ikke Guds <b>Kjerlighed</b> imod os , da han har givet os et saa let Middel til at blive salige ! For os er ingen Slange opreist , men Jesus , den Ophsiede ; hvo , som fluer op til ham i Troen , han flal finde Naade . Dette Middel er ikke for en < rrbar , dydig Mand , som har nok i sig <b>selv</b> ; ikke for en dydig , sat <b>Kone</b> , som tenker at komme i Himmelen , fordi hun opfsrer sig godt , men det er for de Arme , for Syndere , som ikke kunne hjcrlpe sig <b>selv</b> , hvis egen Roes , egen Kraft og egen Dyd er forbi , som ikke lengere finde nogen Trost i sig <b>selv</b> , og som torste ester Naade , ester Barmhjertighed . For Saadanne er det , for Alle uden Undtagelse , enten de ere Christne , Isder eller Hedninger ! Alle de , som troe paa den Korsfestede , stulle ikke fortabes , men have det evige Liv . I Orkenen blev Kobberflangen opreist paa en hoi Stang , paa det alle Israeliter kunde see den , om de ogsaa vare bidte i den fjerneste Dccl af hele Leiren . Saaledes er ogsaa Jesus Christus ophsiet for Alles Sine ; Alles Sine stulle see ham . I de fjerneste Dele af Jorden stal det blive viist den arme Menneflehed, hvorledes Christus hang paa Korsets Troe for Alle som det reneste Forsoningsoffer ; det flal predikes over den ganfle Verden , forkyndes med Herolders Stemme , paa det at alle Mennesker i Troen maae flue op til denne Korsfaestede , som til deres blodige Borgen og Forleser ; Hovedindholdet af enhver Prediken af det guddommelige Ord flal vere : Jesus , den Korsfestede ! — Ak , hvor snflede jeg ikke , at jeg havde den Naade , at fremstille Chnsti Kors saaledes , at det blev seet af Alles Dine !  leg flammer mig ikke ved Christi Evangelium " — siger Apostelen Paulus —  om det end er denne Verdens Vise til Spot , Isderne en Forargelse og Grekerne en Daarlighed ; vi predike alligevel den korsfestede Christum , Guds Kraft og Guds Viisdom " ( 1 Kor . 1 , 20 — 24 ) . Derfor siger han :  leg agter mig ikke at vide Noget iblandt Eder , uden lesum Christum og ham korsfestet , " derfor siger han til Galaterne ( 3 , 1 ) , at han havde malet Christus for deres Sine , som om han var korsfestet iblandt dem . Dette skulde vere Hovedindholdet af alle hans apostoliske Predikener , for at

<h4>2662</h4>Men fornemmelig er dette ydmyge , bsiede , smeltede Sind en Egenflab hos den nve Pagts Born . Den store , hellige Apostel Paulus bekjenver om sig <b>selv</b> :  Han er denstsrste llandt alle Syndere; men ham , den ringeste blandt alle Hellige , var Barmhjertighet» vederfaren " ( 1 Tim . 1 , 15. 16 ) . Hvor liden er dog ikke her den store og hellige Paul us , og hvor stor forekommer ikke Gud ham , i sin <b>Kjcerlighed</b> og Barmhjertighet ) ! Vi maa ikke troe , at Paul us har sagt dette saaledes som naar man udtaler nogle intetsigende , udenadlcrrte Ord ; nei , dette udsprang af <b>Sandheden</b> , af hans Aands dybeste Folelse ; thi han betragtede sig virkelig som den ftsrste af alle Syndere . O , hvor saare bestjcemmer ikke den hellige Paul us de vanhellige Christne , som stedse kun have Noget at dadle hos Andre , og holde sig for bedre end dem , ncere store Tanker om sig <b>selv</b> i sit <b>Hjerte</b> og have de meest overdrevne Indbildninger om sin egen Person . De anste sig for ftomme , for gode og fortreffelige Mennesker , og Paulus , den hellige Paulus , der gjennemreiste Lande og Have , for at vinde Sjcele for Frelseren , Paulus , der kunde sige :  Guds <b>Kjcerlighed</b> er udsst i vore Hjerter " ( Rom . 5 , 5 ) ; der kunde trcede frem og sige :  Alligevel lever jeg , dog ikke jeg mere , men Christus lever i mig ; men hvad jeg nu lever i Kjedet , det lever jeg i Guds Sons Tro , som elskede mig " ( Gal . 2 , 20 ) ; — Paulus , der af idel Kjærlighet » med Glcede hengav Legeme , Sjal og Liv og Alt , hvad han havde , for Lammet , Paulus erklærer offentlig og vidner for hele Verden , ikke med fremhyklet Ydmyghed , men for fuldt Alvor :  leg er den stsrste blandt alle Syndere . " Hvo holder sig nu til <b>Sandheden</b> ? Paulus eller de egenretfcerdige Christne ? Hvo har den Helligaand ? Paulus eller de ? De store Tanker , som et Menneske har om sig <b>selv</b> , de kunne nu angaae , hvad de vil hos Mennesket , dets Forstand , dets Krast , dets Fromhed , — alle saadanne Tanker ere en Vederstyggelighet) for Herren , som alene er ophsiet og for hvem Alt er som et Intet ; de ere ham mere forhadte end de fleste andre Synder : Hoer og Mord og Tyveri ere ikke saadanne Vederstyggeligheder i lehovahs <b>Oine</b> som Hoffcrrdighed i en Synders <b>Hjerte</b> ;  thi " — siger Apostelen —  Gud sta aer de Hoffcerdige imod . " Om et Menneske sidder fast i en Synd , det vcere hvilken det vil / saa kan han dog i et beleiligt Dieblik endnn gribes af Herren og reddes som en Brand af Ilden ; men saaledes er bet ikke med den Hoffcerdige ; denne maa Gud staae imod . Men hvor skrcekkeligt er det ikke , naar Gud staaer et Menneffe imod ! Almindeligviis er kun Mennesket Guds Fiende , og Gud er Synderens Ven og soger hans Salighed og har Medynk med ham ; men saaledes er det ikke med den Hoffcerdige . Imod denne

<h4>2725</h4>ledes som Gud , og Gud er <b>Kjcrrligheden</b> , og Herren har indrettet denne Huusholdning , og Herren er Gud ! Og hvad stal jeg sige om den nye Pagts Dage . — Efterat Gud fordum havde talet mange Gange og paa mange Maader til Faedrene ved Propheterne , saa haver han i disse sidste Dage talet til os formedelst Ssnnen ( Ebr . 1 , 1 ) . — Dette er en Forananstaltning, som allerede bcrrer det Guddommeliges Kjendetegn og Praeg i sig . Sode Evangelium , hvor ofte er Du ikke bleven nedsat , og nedtraadt i Stevet af Svinene som en Perle , hvilken de ikke kjendte ! Korsfestede <b>Kjcerlighed</b> , hvor ofte er Du ikke bleven miskjendt , hvor ofte er Du ikke bleven bespyttet paany og korsfestet ! Evige Viisdom , hvor ofte har Du ikke allerede maattet lade Dig mestre af dine egne Born , ja af Daarer og Spottere ! Desuagtet er og bliver dog det , som Ssnnen har givet os , det Guddommeligste , som Menneskeheden eier ; desuagtet er dog den paa Korset ophsiede Jesus den sande Gud og det evige Liv , og det af Aanden aabnede Vie betragter ham dog som Saadan og kjender ham som Saadan ; desuagtet er dog Ordet om Korset Guds Kraft og Guds Viisdom ; desuagtet er dog Ingen endnu , som betragter Evangeliet og virkelig har forsket efter <b>Sandheden</b> , forbleven i Uvidenhed , men er kommen til den evige , klippefaste <b>Sandhed</b> , til Ja og Amen i sit <b>Hjerte</b> : Ja , Amen ! Den nye Pagts lehovah , Jesus Christus , er Gud ! Han alene fortjener , at man elsker ham af ganske <b>Hjerte</b> ; han alene fortjener , at alle Kna bsie sig for ham , og at alle , alle Tunger stal bekjende , at han er en Herre til Gud Faders Mre ( Phil . 2 , 11 ) ! — Og , hvad mene I ? Dersom det ikke var af Gud , vilde det da vel vere bleven staaende indtil nu ? Vilde da dette Verk endnu vare ved ? Vilde det ikke vaere bleven kvalt i sin forste Spire ? Sandelig , den christne Kirkes fortsatte Bestaaen gjennem alle Tider indtil denne sidste bedrsvelige Tid er Vidnesbyrd nok om , at « Herren , han er Gud , " at han er den sande Gud og det evige Liv . Betragter den ringe Begyndelse , af hvilken dette Vcerk er udgaaet ! En übekjendt , fattig Tsmmermand fra Nazareth begynder at predike i Herrens Navn og forsamler om sig <b>selv</b> fattige Mend , « vidende , ulcerde Folk ; han vandrer omkring i Fornedrelse og en Tjeners Skikkelse ; han bliver ophcengt paa Korset som en Misdeder og dscr , og skjult for Verdens <b>Oine</b> opstaaer han fra de Dsde og udsender sine tolv Disciple i Merden med den Befaling :  Prcediker Evangelium for al Skabningen " ( Marc . 16 , 15 ) ' . — og hvad steer ? Af dette ringe Sennepskorn udvikler der sig et megtigt Tre , under hvis Grene Himmelens Fugle sidde , et Guds Rige , i hvilket de fjerneste Folkeslag og Nationer boe i Fred og nyde af det evige Livs Frugter . Hvo maa ikke

<h4>2872</h4>Eder Liggendefce i Himmelen , hvor hverken Mol , ei heller Rust fordcerver , og hvor Tyve ikke gjennembryde og stjale . Men ssger fsrst Guds Rige og hans Retfcerdighed , saa stulle og alle disse Ting tillcegges Eder . " Hermed henviser han os til det himmelske Sind , som vi stulle lade os stjcenke , grundlcegges og befcestes i os formedelst Guds Aand og Naade . Men hvilket er da dette himmelske Sind , om hvilket vitale ? — Dersom En ssger Guds Rige og hans Retfcrrdighed , dersom En arbeider paa sin egen Saliggjorelse med Frygt og Bcrven , dersom En griber og fastholder det ene Fornsdne , dersom En kan sige med Apostelen Paul us :  Et gjor jeg , forglemmende , hvad der er bagved , og roekkende efter det , som er foran , iler jeg mod Maalet til det Klenodie , som horer til Guds Kald herovenfra i Christo Jesu " ( Phil . 3 , 14 ) ; dersom En , ligesom Enoch , vandrer med Gud , og har sit Borgerstat ) i Himmelen , og ikke lcrnger ssger nogen Glcede i denne Verdens Ting , men higer efter det Uforgjengelige og Evige : — da har han et himmelstt Sind , da opfylder . han Budet i Dagens Evangelium , da ligner han den Mand , ' som byggede sit Huus paa en Klippe , hvor det trodser hver Skylregn og hvert Vandlob og alle Storme . O , et saadant himmelstt Sind er noget Stort , Saligt og Sjeldent ! — At agte det for Intet , som man seer , og at scette Priis paa og elste det , som man ikke seer ; at hange ved Gud i Christo formedelst Troen og den retflafne <b>Kjcerlighed</b> og hverken ved Satans Blcendvcerk , eller ved Verdens Forfsrelser og Tillokkelser, eller ved sit eget fordcrrvede <b>Hjertes</b> og Kjods Drifter og Fristelser , — ikke ved Noget af Alt dette eller ved nogen Ting i Verden at lade sig afvende fra en alvorlig , fast Gang mod Evigheden; at stue hen med ufravendt <b>Blik</b> paa det foreliggende Klenodie, stedse at komme lesum Christum , den Korsfcestede og Gjenopstandne, ihu , og om end Veien gaaer gjennem mange Gjenvordigheder, gjennem mange Forncegtelser , at efterfslZe Lammet , der har elstet os , der har kaldet os , der har opoffret sig for os , — dette er at have et himmelstt Sind , dette er noget overmaade Stort og beroer heller ikke paa nogen menneskelig Magt ; dette er noget Sjeldent , thi Verden , den store Hob , veed og aner Intet deraf . Og dog er denne Vei i stadig Sogen efter Guds Rige den eneste Vei , paa hvilken vor Sjcel sinder Fred , <b>sand</b> Sjaelefted. Thi Gud har lagt Evigheden i Mennestets <b>Hjerte</b> og Sjcelens dybeste Lcengsel kan Intet stille uden Gud alene , den i Christo Jesu aabenbarede Gud , som Verden ikke kjender , men efter hvem dog ethvert Menneskehjerte har et hemmeligt Suk og en staerk Lcengsel i sit Inderste , om det endog er skjult for Mennesket <b>selv</b> , om Mennesket endog er ligesom nedsunket i Syndens Dyno . O ,

<h4>2974</h4>til at blive rene af Hjertet , som elske og soge Freden , som da ogsaa Allesammen lide Forfslgelse for Retfoerdigheds og for Jesu Skyld og lide allehaande Uret — kun disse ere paa den trange Vei , som forer hen til Livet . Men hvo , som ikke soger nogen bedre Retfcrrdighed end de Skriftkloges og Pharisacernes Retfcerdighed, hvo , som ikke elsker sin Broder , hvilken han seer , hvo , som ikke er velvillig mod sin Modstander , medens han er med ham paa Veien , hvo , som bedriver Hoer , om endog kun med Vinene og ikke strar i sit <b>Hjertes</b> Dyb bliver forfcerdet over et saadant <b>Blik</b> , hvo , som ikke overalt soger <b>Sandheden</b> , men endnu er istand til at lyve , til at bande , til at forsvaerge sig , hvo , som har Sind til at gjengjcelde sin Nceste Ondt med Ondt og ikke langt heller lider Uret , hvo , som endnu kan hade sine Fiender , og ikke strceber at elske dem , og ikke beder for dem , — alle Saadanne ere Fordaervelsens Born . Guds Born soge ikke udvortes Anseelse ved sine Gjerninger ; Guds Born ssge fsrst Guds Rige og hans Retfaerdighed, de have ikke et <b>Hjerte</b> , som er deelt mellem Gud og Mammon ; de sattte sin Fortrsstning til den levende Gud ; de kaste sine Sorger paa ham ; de ere altid beskjeftigede med at drage Bjoelken ud af sit eget Vie , og derefter ste de til at uddrage Skjerven af sin Broders ; de ere forsigtige i sit Forhold til Verden , de have Bonnens Aand og lade i alle Ting sine Begjæringer fremfsres for Herren i Paakaldelse og Bon ( Phil . 4 , 6 ) ; de ssge at elske sin Naeste som sig <b>selv</b> . Se , det er den trange Vei , som forer til Livet ! Nu kan Enhver herefter prove sig og ste , om han befinder sig paa denne trange Vei . Det har kostet mig stor Overvindelse at troe og proedike , at denne er den trange Vei ; jeg vilde heller have sogt og sundet den paa et andet Sted . Hvor gjerne vilde jeg ikke gjore Veien bredere og mageligere for Eder og for mig ! Hvor gjerne vilde jeg ikke sige til Eder og til mig <b>selv</b> : Kjaere Venner ! vi behove ikke at domme os saa strcrngt , Frelseren tåger det ikke saa nsie ! Men jeg tor ikke og kan ikke ; det staaer engang saaledes i Guds hellige Ord , og ved dette Ord vil det blive , saa vist som Frelseren har sagt :  Fndtil Himmel og Jord forgaae , stal ikke end een den mindste Bogstav eller een Tsddel forgaae af Loven , ftrend de Ting skee allesammen " ( Matth . 5 , 18 ) . Hvad er her at gjsre , mine kjcere Tilhorere ! Alle onste vi at blive salige . Ikke <b>sandt</b> ? I Mcrnd , I <b>Kvinder</b> , I Ugifte , I Letsindige ! Er det ikke <b>sandt</b> , I onste dog ogsaa at blive salige ? Ikke <b>sandt</b> ? I Gjerrige ! Ikke <b>sandt</b> ? I Hoerkarle og Skjorlevnere ! Ikke <b>sandt</b> ? I Bedragere , som finde den stsrste Glcrde i at kunne tåge en Skilling mere fra Eders Naeste , end han er styldig ! I voldsomme , hovmodige , jordifltsindede Mennesker ! Ikke <b>sandt</b> , kjare Tilhorere og Medskyldige for Gud ! Vi emste dog Alle at blive

<h4>3085</h4>Men , hvad grced Han over ?  Der han saae Staden , grad han over den . " Kjere Tilhsrere ! Naar vi grede , saa flyde vore Taarer fordetmeste over os <b>selv</b> og over Ting , som have Hensyn til os <b>selv</b> . Naar nogen Ting gaaer os imod , naar vel udtenkte Planer blive gjennemkrydftde , naar vi synes , at det gaaer os haardt , naar Lidelser komme eller et eller andet Tryk : da flyde vore Taarer . Det haardeste Menneske kan tilsidst blive bragt til at grede , naar han angribes af Gud paa sin emme Side og kastes ind i Trengselens Ovn . Der findes ogsaa blsde Gemytter, som strar kunne blive rotte til Taarer ved Fortellinger eller andre Fremstillinger , eller ved Synet af en Lidende . Men da gaaer det ikke dybt ; et Sieblik cfter kunne de lee paa samme Maade , som de hidtil have grcedt , disse Taarer ere ofte ikke Andet end Fslgerne af ophidsedc Nerver eller en opfiammet Indbildningskraft, Vidnesbyrd om en uordentlig Sindsforfatning . Saaledes har ikke Frelseren grcedt . Han grced ikke over sig <b>selv</b> . Han kunde vel have gjort det , hvem stulde derfor have forarget sig over ham ? Nu red han igjennem Jerusalems Porte ; bag ham laae de treogtredive Aar af hans moisommelige Kamp paa Jorden ; saa mange Ydmygelser , saa meget Bittert havde msdt ham ; saa mange tunge Trin havde han gjort gjennem denne usle Verden ; foran ham laae en skrekkelig Fremtid , hans sidste Kamp , hans forsmedelige Rdelse og Dsd ; Alt dette laae aabent for Frelserens Sjels <b>Oie</b> ; han saae de Steder , hvor han stulde opleve sin dybeste Fornedrelse ; han saae Pinselsstedet for sig — hvo kunde have forarget sig over ham , hvis han var bleven blod om Hjertet , hvis han havde gredt over sig <b>selv</b> , over sit lange Lidelseslsb, der stulde ende paa en saa skrekkelig Maade , hvis den Tanke var ovstaaet hos ham : Ak , Du arme Slagtsffer , her stal Du hoste den blodige Lem for din store <b>Kjerlighed</b> ! Dette vilde i <b>Sandhed</b> ikke have veret nogen Svaghed . Men derover grced han ikke . s , Der han saae Staden , gred han over den . " O , mit <b>Hjerte</b> , tab Dig ganste i Jesu <b>Hjertes</b> Storhed ! Fol tilfulde , hvilken Frelser , hvilken Yppersteprest Du har ! Jerusalem, den store Synderes Stad , laae for hans Blikke . Gud havde i lang Tid gjort usigelig Meget for denne Stad og for Folket i denne Stad . Han havde givet dem sin Lov ; han havde sendt sine Tjenere , Profeterne , til dem ; han havde hast usigelig megen Taalmodighed med deres Svagheder og Ondstab ; han havde

<h4>4568</h4>et , forlader de nioghalvfemsindstyve i Brkenen , og gaaer bort efter det tabte , indtil han sinder det ; dette et hans <b>Hjertes</b> vigtigste Anliggende . Men en anden , ligesaa stor Omsorg af den gode Hyrde er den , at den , som engang er grebet af ham , ikke paa nogen som helst Maade skal blive reven ' ud af hans almegtige Hyrdehaand , men at hans Faar maa blive bevarede fra det Onde indtil hans Tilkommelses Dag . Hvor inderlig har ikke den store Yppersteprest i sin sidste Bon anbefalet sine Disciple og sine Troende til Faderen :  leg beder ikke , at Du stal tåge dem ud af Verden , men at Du stal bevare dem fra det Onde . De ere ikke af Verden , ligesom jeg er ikke af Verden . Hellige Fader ! Hellige dem i <b>Sandhed</b> ; dit Ord er <b>Sandhed</b> " ( Joh . 17 , 15 — 17 ) . O , hvor udgaae ikke disse Ord fra den trofaste Yppersteprests <b>Hjertes</b> inderste , dybeste Grund , han , som med uendelig <b>Kjerlighed</b> omslutter og omfatter Sine , og hvor fremvelder ikke deraf det varme Vnfle , at see Sine urokkeligt befcestede i <b>Sandheden</b> og bevarede i hans Haand , uden at rives ud af den ! Ligesom hans <b>Hjerte</b> flog og brendte af <b>Kjerlighed</b> til hans Disciple , saaledes brcender det endnu lige til vore Tider for alle hans Troende , ja endog for den ringeste af hans Brsdre . Derfor luede ogsaa i Apostlernes Hjerter en saa hellig Flamme af den inderligste <b>Kjerlighed</b> til Brsdrene ; <b>Blikket</b> i Jesu kjerlighedsfulde <b>Hjerte</b> havde antamdt den hos dem . Derfor lcrse vi i de apostoliske Breve , at P au lus stedse i sine Bsnner tenker paa Menighederne , at de skulde megtigen bekreftes i Henseende til det indvortes Menneske , rodfestes og grundfestes i <b>Kjerlighed</b> , at de skulde opvoxe til ham , som er Hovedet , til Christus ; derfor advarer han dem som en Fader og Broder fra Affald , fra Utrostab , fra Vantro , fra Forfsrere og Vranglcerere ; derfor skriver han til Ebrceerne : , Seer til , Brsdre ! at der ikke i Nogen afEder skal vere et ondt , vantro <b>Hjerte</b> , saa at han affalder fra den levende Gud " ( Ebr . s , 12 ) . Det Samme laa ogsaa Apostelen Johannes paa Hjertet, da han strev til sine Lesere :  Mine Born ! bliver i ham , at naar han aabenbares , vi da kunne have Frimodighed , og ikke stulle blive bestjemmede af ham i hans Tilkommelse " ( 1 Joh . 2 , 28 ) . Det var Apostelens <b>Hjertes</b> inderligste Lengsel , at de Sjele , som engang vare grebne af Jesus Christo , ogsaa maatte holde trofaste ud indtil hans Tilkommelse . Men hvorfra havde de denne inderlige <b>Kjerlighed</b> , uden fra Herren <b>selv</b> ? O , kjere Tilhsrere ! Frelserens <b>Hjerte</b> banker af <b>Kjerlighed</b> til hans Troende ; han har hengivet sit Liv i Dsden for dem , han har udsst sit Blod for dem , <b>Kjerligheden</b> har drevet ham ind i den dybeste Dsdsnsd , har gjort ham til et Offer for dem ; derfor havde han heller intet hsiere Onste , end at frelse Sine , at gjsre Sine evig salige , og

<h4>4792</h4>stor Familie , Bsrnebsrn og Bsrnebsrnsbsrn , som holdt den gamle bedagede <b>Moder</b> hsit i ) Ere , beredte hende mangen Vederkvagelse , Nydelse og Glade , saa har det dog ikke , som man vel kan tanke , manglet paa nedflaaende Omstandigheder , til hvilke navnlig Horte , at hendes elskede Datter for ikke et Aar siden afgik ved Dsden , hvilket dybt angreb og bedrsvede hende og , som det synes , bidrog meget til at vende hendes <b>Blik</b> opad mod Himmelen . Og saa har hun da i sin sidste Sygdom , ved hvilken hendes Krafter langsomt bleve fortarede , overgivet sig med stor Ro og Hengivenhed i sm Guds Villie , som hidtil havde elsket og fort hende saa vel , og ogsaa hjulpet hende ud af saa Meget og aldrig forssmt eller forladt hende ; nu er hun , som vi haabe til Frelseren og hans <b>Kjarlighed</b> , trangt md i Frihedens Rige , hvor ingen Alderdom gives langer , men hvor hans Gjenlsste i evig Ungdom og Fornyelse nyde Fred og evig Fryd af Christi Fylde . Skulde jeg beskrive hendes aandelige Tilstand , saaledes som den aabenbarede sig paa hendes sidste Sygeleie , saa maatte jeg dertil bruge lakobs Ord , naar han siger :  Herre , jeg bier efter din Salighet » ! " Da han nemlig tog sine Fsdder til sig op paa Sengen og var i Begreb med at udgive sin Aand , saa sagde han endnu , midt under de Velsignelser , som han , den gamle , udlevede Guds Vandringsmand , gav sine Sonner :  Herre , jeg bier efter din Salighed ! " Han havde en Lobebane bag sig , om hvilken han <b>selv</b> havde sagt , at den havde varet kort og flem ; gjennem mangen Angest og Nsd havde han vandret , og hans Sind var derunder bleven bsiet og ydmyget ; en fast Tro paa hans Fcedres Gud var derunder bleven indplantet og bevaret i hans <b>Hjerte</b> ; derfor oplsftede han sit <b>Blik</b> , da han skulde trade ud af denne Jammerdal , og det vederkvcrgede ham , naar han tankte paa den Guds Salighed, som skulde aabenbares for hans Efterkommere , og hvorved der skulde beredes dem Trost og Glade paa denne af Herren forbandede Jord , paa hvilken der kun findes Ondt og Kummer , og hvor nu den store Lov gjalder i Bsrnenes Opdragelse , at de kun gjennem mange Trangfler kunne indgaae i Guds Rige . I hans Dsdstime var det ham til stor Bederkvagelse , at han kunbe stue hen paa Guds tilkommende Salighed , paa Israels Trost , paa lesum Christum og paa den med ham frembrydende Forlssningens Dag , og at han turde nedsanke alle sine onde Dage og alle sit <b>Hjertes</b> Bekymringer og Angester i dette store Haab , og hvile sig i det . I dette Guds Lys forberedte han sig trsstig til at overgive sin skrsbelige Hytte til Dsden . Ogsaa vor hensovede Medssster har med <b>Blikket</b> paa den Evighed , som hun gik imsde , med Ro og Hengivenhed ventet pc ! a den Salighed , som i Christo er opgangen for Verden , og som

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009072703031'target=_>Hofacker, Ludwig, 1863, Prædikener paa alle Søn- og Helligdage</a><h4>97</h4>Stuttgarts Indbyggere have mig til Diakonus ved St . Leonhard , og bestormede Kongen med Bonner og Underskrifter . Imidlertid faldt jeg ien svag Sygdom , som efterhaanden forbandt sig med den tidligere Svaghed i mine Hjernenerver , saa at jeg fra Februar 1825 til Februar dette Aar ikke kan mindes at have lest nogen Bog uden det guddommelige Ord og nogle korte Brcve . Om et Arbeide i Herrens Viingaard var der desuden ikke Tale ; jeg var igjen sat paa Pension af Frelseren . Jeg blev sorgmodig , ogsaa med Hensyn til mit fremtidige Udkomme , uden Formue , nden Lon , uden Embede , og given til Priis for en Sygdom , som , efter tidligere Erfaring kunde vare mindst to Aar , med min <b>Moder</b> og min sindssyge Broder hos mig , der for en Deel ventede Midlerne til sit Ophold hos mig . Desuden skulde jeg bruge et Bad ved Siden af andre Kure . Udsigten var ikke behagelig ; men Herren sorgede for os . Han brugte Vennerne i Stuttgart som sine Redstaber , og jeg stal tåge min Tak til disse Folk med mig over i Evigheden . Paa en meget ftin og skaansom Maade fik vi os stedse paa de meest forskjellige Veie og fra skjulte Venner tilsendt saa Meget , at vi kunde leve uden Sorger og tillige bruge de nodvendige Kure . I August 1825 kom jeg tilbage fra St . Moritz , og ventede nu i Stuttgart paa en Ansettelse , blev syg , kunde ikke lese , ikke studere , haabede og haabede , hvor Intet var at haabe , og satte min Fortrøstning til Herren , under megen Vantro , mange Tvivl og andre onde Tanker . Saaledes vedblev det til Slutningen af Oktober , da det behagede Gud at hjemsoge mig med den haardeste Sygdom , som jeg nogensinde har oplevet . Fra November af kjender jeg ikke Andet end usammenhengende Dromme og Phantaster . Det var Nervefeber i sterkeste Grad , og jeg var allerede ganske ovgivet af Legerne. Men min trofaste Jesus , som ikke vilde at jeg skulde fortabes , og som sogte min Sjel med usigelig Taalmodighed , reddede mig ogsaa af denne store Trengsel . Hvad de trofaste Brodre i Stuttgart uuder denne Tid have gjort imod mig , hvorledes de uden at vemmes have holdt mig , vaaget over mig , betjent mig med lisomslag , baaret mig , lagt mig og ellers beviist mig sin Troskab : dette kan jeg aldrig forglemme . Frelseren, der gjengjelder en Draabe Vand , som man rekker en torstig Discipel, ville optegne dette og gjengjelde Brodrene det . Efierat jeg forgjeves havde meldt mig til Stammheim ved Ludvigsburg , sogte jeg siden Nielingshausen og blev ogsaa udnevnt dertil . Jeg onskede endnn at predike i Stuttgart , men da dette blev mig fraraadt af anseete Folk , afstod jeg derfra . Hvad jeg i denne Tid har erfaret om Guds bestemte og omhyggelige Vaagen over sine Born , hvad jeg har erfaret om Hans Veie , der " snart kommer med <b>Kjerlighed</b> , snart med Lidelser , dette ( hvoraf jeg her har givet Eder ligesom kun Rammen af Maleriet ) , dette skal , som jeg haaber og beder , blive til evig Tid indprentet hos mig . "  Nermere kan man vel ikke staae Evigheden , " fortsetter han endvidere i et andet Brev fra denne Tid ,  end jeg har staaet den under Nervefeberen . Men hvor bliver da Takken for at Herren lod mig usrugtbare Tre blive staaeude endnu lenger ? Ak , min Frelser ! hvor bliver der da af Takken ? Skal da mit haarde <b>Hjerte</b> ikke ved Velgjerninger, ikke i Gjenvordighedeus Ovn , ilke ved nogen Ting kunne blive bragt til Opmerksomhed ? — Jeg har gjort den Erfaring , at Intet forsiaaer til at bringe et Menneske endog kun til en eneste <b>saud</b> Fslelse angaaende sig <b>selv</b> og sin Gud , Intet uden Christi Blod . Cbristi , Guds Lams Blod maa til , — og , o evige , paa Korset naglede <b>Kjerlighed</b> ! — saa haard er jeg , og saa mild og naadig er Du , at Du , da Du

<h4>754</h4>vilde sende sin Helligaand , at han med sin levende Griffel uudstettelig maatte indprcrge det store , det sode lesunavn i vort stakkels <b>Hjerte</b> ! Dette kunde jeg behove ; thi dersom dette ikke foregaaer i mit <b>Hjerte</b> , saa er jeg kun en lydende Malm og en klingende Bjeldc . Dette kunde Loe r erne ved vore Skoler bchsve, for at de maatte vcere istand til at fore de dem af Herren anbetrocde Lam p , , a de rette GrEsgange . , Dette kunde vore Embedsmcrnd behsve , for at de kunde styre sit Embede ester Frelserens Sind . Dette kunde vi Alle , Alle behove , for at vort Sted endelig engang maatte ophore at verre et Hjem for urene Aander , for at Guds Rige kunde bryde frem overalt , i Husene , paa Gaderne, paa Markerne og i Skovene og komme tilsyne med Rctflrrdighed, med Fred og Gladde i den Helligaand . Betccnler , kjcere Tilhsrere ! vor Tid iler fremad , vi ere igjen et Aar crldre . Ak , vi have forgjeves levet , fredet , arbeidet og bemoiet os ; Alt dette er til Intet , ja mindre end Intet , dersom ikke Icsu Navn bliver forklaret i os ved den Helligaand , dersom vi ikke vandre gjennem Verden i Guds Sons Tro og <b>Kjcerlighed</b> . Jeg opfordrer Eder , I Verdensmennesker , som <b>selv</b> bekjende , at I ere det ; I , som vilde optage det overmaade ilde , dersom man regnede Eder blandt de Fromme , altsaa I gudlose Mennesker, ester egen Tilstaaelse , enten I nu ere 20 , 30 , 40 , 50 , 60 , eller 70 Aar gamle , siger stlo : Hoad have I havt af denne Levetid? Have I oafta havt nogen <b>sand</b> Glcede ? Have I ogsaa havt nogen <b>sand</b> Nydelse ? Haoc I nogensinde smagt nogen af Syndens Nydelser , uden der var Bitterhed deri ? Visselig ikke ! Nu , hvilken Frugt have I da havt af Eders Liv til denne Stund ? Svaret bliver , at det er faret forbi som en Tanke ; at der opstaaer stor Uro i Eders Hjerter ved at kaste et <b>Blik</b> tilbage og ved at skue hen imod Evighedm ; at I maaskee med Kummer og Sorg ere indtraadte i dette nye Aar , og maaskee ere blevne gamle og graa , fsr I have oplevct noget Godt . Ak ! et Menneskeliv er saa gcmske tomt og vcerdlsst , naar det ikke bliver fort i Samfund med Frelseren . Vinter og Sommer , Dag og Nat kommer og qaaer , man arbeider i sit Haandvcrrk , i sit Kjsbmandskab , paa sin Ager . man vil erhverve Noget og man kan ilke ; man vil i det Mindste gjore sig gjceldfti , men atter gaaer det ikke ; imidlertid indsmder sig megen Kummer og Elendighed , mange Sorger og Synder ; der findes Intet mere elendigt og jammerligt , end et saadant Liv ; man er intet Andet end et Lastdyr . Men hvo der kjender lesum . han gjennemgaaer den ncervcerende Tid og veed ogsaa , hvorfor han er her paa Jorden . Og tilsidst bortkaster man de udstidte Sko fra denne forgjcengelige Levetid , og farer til lesum , hvem Sjcelen elsker og

<h4>975</h4>sit Foder . — De Eftertamkende iblandt os spsrger jeg : have ikke i det mindste Nogle blandt Eder bidtil taenkt sig Frelseren saaledes , at han har vceret en Dreng i Henseende til Legemet , men ikke i Henseende til Aanden ? Men , seer , deri tåge I hsiligen feil . Jesus var i sit tolvte Aar en Dreng , saaledes som vore tolvaarige Drenge ; han maatte vore paa Legeme og Aand , som vi maa voxe ; han maatte loere , som vi maa laere . Han vidste ikke Alt iforveien , det see vi i Dagens Evangelium , som beretter , hvorledes han i tre Dage laerte l Templet ; han sad der midt iblandt Folkets Lcerere for at hore paa dem og spsrge dem ad ; det var ham om at gjore , at lcrre Noget af dem . Vistnok kan man ikke ncegte , at der allerede i denne Barnealder har fremlyst og talt noget Sceregent ud af hans Person ; en moerkelig , for Drenge i hans Alder usedvanlig Viisdom , Enfoldighed og Kj ^ rlighed til <b>Sandhed</b> , Indblik i <b>Sandheden</b> , som blot kan forekomme hos et Menneske , der ikke , saaledes som vi , er undfanget og fsdtiSynd , hos hvem ingen skjult Lyst eller <b>Kjcerlighed</b> til Synden kunde forvende Biet fra dets Enfoldighed og Klarhet » og fordunkle <b>Blikket</b> i <b>Sandheden</b> . Men af guddommelige Egenskaber var der Intet at see ; han havde udwmt sig fra dem , han havde skjult dem for sig <b>selv</b> og for Andre ; Frelseren var nu netop en Dreng og besad alle en Drengs svrige Egeufkaber , med Undtagelse af Synden — en hellig Dreng , men dog en Dreng . Kjcere Tilhsrere ! lader os nu betragte Guds Ssn paa hans forskjellige Udviklingstrin ! Det maa forudsackes , at Guds Ssn af <b>Kjcerlighed</b> til den faldne Menneskeslcrgt havde besluttet at blive Menncste , at ifsre sig vort arme Kjod og Blod . Denne Beslutning fomdsat , havde han strax som et fuldkomment Menneske kunnet indtrcede i denne Verden ligesom Adam , ikke som Barn , men som Mand . Men delte vilde han ikke , dette laa ikke i Faderens Raadstutning . Han besluttede at gjennemgaae alle Menneskehedens Udviklingslrin ; han vilde i alle Ting voere sine fortabte Brodre liig ; han vilde fsdes som vi , blive et Barn som vi , blive en Dreng som vi , blive en Dngling som vi , blive en Mand som vi ; han vilde gjmnemleve Tiden for en heel Menneskealder paa denne Jord : thi til en Menneskealder regner man treogtredive Aar og lidt over , og saalcenge levede Frelseren netop her i Verden , paa det at han kunde blive en i Alt vel forssgt Ippersteprcest , der kunde have Medlidenhed mel » vore Skrsbeligheder, med vor Barndoms Svaghed , med vor Svaghed som Dreng eller Pige , med vor Svaghed som Ingling eller Jomfru , med vor Svaghed som Mand eller <b>Kvinde</b> , fordi han skulde bcere alle Alderes og alle Sloegters Synder ; fordi han skulde erhverve Kraft for alle Aldere og alle Slcegter til at verre i denne Verden,

<h4>1532</h4>Vand til Viin o . f . v. Videre end til saadanne ydre Kjendsgjerninger naae ikke lettelig de Fortellinger , som gaae i Folkenmnden. Men dermed kunde den kananeiste <b>Kvinde</b> ikke finde.sig tilfredsstille ; hun forhorte sig videre ; hun hsrte , at han nedstammede fra Davids Slc ^ gt , og maaskee var den forjettede Davids Son ; hun sogte Underretning om hans Opfsrsel , om hans Hjertelag, og horte om hans <b>Kjcerlighed</b> til de Elendige , om hans store Barmhjertighet » . Og hvad hun saaledes hmte om ham , det frembragte hos hende en übegrendset HsiagtM for Jesus og en urokkelig Tiltro til hans <b>Hjerte</b> . - O , mine kjere Tilhsrere ! Hvad stulle vi sige om denne hedenske <b>Kvinde</b> , som saaledcs annannnede og benyttede de ringe eg upaalidelige Etterretninger , hun havde om Jesus , at den stauke Tro , som Frelseren <b>selv</b> beundrede hos hende , kunde voxe frem af dette lille Frokorn ? Hvad stulle vi sige om denne <b>Kjcerlighed</b> til <b>Sandheden</b> , om denne Trostab i at bevare , hvad hun havde modtaget? Sandel ig ! Denne <b>Kvinde</b> er sat som et lysende og fordsmmende Tegn for os og denne Tidsalders Mennesker . Mange Hedninger , som dog ikke kjende Guds Vei , vise for ncervcerende den storfte Begjerlighed ester Guds Ord og ester Lcerere . Folk , som fsrst for kort Tid siden have lert Evangeliet at kjende , lere med den stsrfte Begjcrrlighed at lese , for at kunne lese Guds dyrebare Ord ; de agte Guds Ord for sin dyrebareste Skat ; de kunne ikke blive mette af at hore om sin Jesus ; de soge Frelseren ; de soge hans Ansigt ; med mange Bonner og Taarer ssge de Guds Naade og sine Synders Forladelse i Lammets Blod . Herrens Herlighed opgaaer over mange Hedninger , og vi ste dem vandre i dens Lys . Vi derimod have Guds Ord i saa rigeligt Maal ; vi have det hjemme og i Kirken , og agte denne dyrebare Skat som noget Almindeligt , og det forekommer os , at det saa maa va > re . <b>Sandheden</b> , det evige Evangelium have vi i saa rigeligt Maal , og vi rsre os ikke , og de Fleste blive i sine Synder, i sit Liv ester Fedrenes Viis , i sin aandelige Uforstand og Blindhed , og takke ikke for Guds ufortjente Forbarmelse , oH tenke ikke paa at takke . Hvad vil dog Gud maatte- foretage sig med os ! Naar han lader sit dyrebare Evangelium predikes for os ; naar han lader Christum afmale for vore Bine ; naar han lader et saa rigeligt besat Bord fremsette for os , og vi ikke aabne Munden for at nyde , men for en Deel endog vemmes ved denne Spise , medens mange Tusinde vilde vere glade ved de Brsdsmuler, som falde fra vort Bord ; hoad vil da Gud blive rwdt til at foretage med sin dovne , kolde , mette , stolte Christenhed ? Det vil han maatte foretage med den , at han aldeles omstoder dens Lysestage , hvortil d ? r i vor Tid allerede er Udsigter , saa at

<h4>2276</h4>Stemme er trcengt ind til Eders <b>Hjerte</b> og har overbeviist Eder om Eders usalige <b>Hjertes</b> Tilstand , om ikke Ordets Stemme allerede magtigt har gjenlydt i Eders Hjerter , saa at I maatte tilsiaas: Saaledes kan saakdes tsr del ikke blive ; det maa blive anderlcdcs med mig . Mange Mennesker blive jo ved Ordets Kraft saaledes rystede og bevcegede , at man jknlde tamke , at det var umuligt Andet , end at de maatte blive nye Mennesker , at <b>Sandheden</b> maatte overvinde Lognen i dem . Men hvorfor skeer dette da alligevel saa sjeldent , hvorfor kommer det saa sjelden til en fuldstcendig Forandring , hvorfor hore saa Mange Ordet og blive alligevel uomvendte i sit <b>Hjertes</b> Gnmd , alligevel übenaadede , alligevel uden Samfund med Christus , stjont Frelsen bliver lagt Enhver saa ucer ? Man lader rigtignok denne eller hiin stette Vane fare , man bliver mere tilbageholden og stille , men suldsicendige Omvendelser forekomme sjelden ; man bliver dog med al Horelse af Ordet en ded , kold , forhcerdet Synder . Og dog vit Frelseren have noget Rigtigt og Heelt , ingen halv Hjertets Forandring , ingen halv Gjenfodelse , han vil have et Folk , som ganske tjener ham , ganske er hans Eiendom ? Halvdelen vil han udspye as sin Mund ' ( Aab . 3 , 16 ) . Hvori ligger da Aarsageu til denne Halvhed ? Ligger den i Herren <b>selv</b> ? Nei , thi den gode Hyrde gaaer med samme <b>Kjcerlighed</b> og Iver ester alle sine fortabte Faar ; han soger stedse at forsamle Sine , ligerviis som en Hone samler sine Kyllinger under sine Vinger ; hos ham ligger Skylden ikke . Hos hvem ligger den da ? Aabenbart hos Mennesket selo ; dette tillukker <b>selv</b> sit <b>Hjerte</b> , det er Menneskets egen , frivillige Gjerning , han vil ikke , han undviger ; han vil ikke lade det komme til nogtt Ret med sig . Hos Mennesket ligger Skylden ; derfor kunde Herren bebreide Isderne:  I ville ikke " ( Matth . 23 , 37 ) . Men , hvorfor vil da Mennesket ikke ? Hvad hindrer ham da ? Forskjellige Ting kan blive til saadanne Hindringer . Hos Mange er det Hovmode n . Jeg har ovenfor sagt , at Guds Ord er et Speil og en Dommer over Hjertets Tanker og Raad . Naar nu en hovmodig Aand kaster et <b>Blik</b> ind i . dette Ordets Speil , og bemaerker hos sig allehaande Ureenhed , Fordervelse og HMighed , saa trcrkker han sig tilbage i sin naturlige Hovmod og sporger sig <b>sclv</b> : Saaledes skulde Du voere beskaffen ? Dette er itke for dm Dannede , det er kun for den raae Hob ; jeg har jo dog Dannelse; jeg har indprceget mig gode Grundscrtnwger . En Anden seer mere paa sin Vsrdighed , paa sit Embede , paa sin Rigdom og paa mere Saadant , og indrommer ikke Guds Ord nogen Ret over sig . Folk af lavere Stand have igjen andre Undskyldninger af Hovmoden : Jeg gaaer i Kirke ; Ingen kan beskylde mig for ,

<h4>2586</h4>Mange sig ! De stue vel ogsaa op til lesum , og troe paa den Korsfachede , men ikke med aandelige Bine , ikke med et saadant <b>Hjerte</b> , som den Helligaand kan oplyse . De ville det ikke ; de troe , at de kunne hjcelpe sig <b>selv</b> , kunne gjsre sig <b>selv</b> vise , og saa blive de dsde i sine Synder . Men den , der skuer op til Frelseren med Bine , som den Helligaaand stjamker , han kan ikke lcengere forblive dsd ; den , som i Aand og <b>Sandhed</b> gjor denne Erfaring: Den korsfcestede <b>Kjcrrlighed</b> er min ! han har det evige Liv ; og dertil naaer ingen Naturkraft , det er Guds Gave . — Men , kjacre Tilhsrere , hvor stor er ikke Guds <b>Kjoerlighed</b> imodos , da han bar givet os et saa let Middel til at ' blive salige ! For os er ingen Slange opreist , men Jesus , den Ophoiede ; hvo , som stuer op til ham i Troen , han skal sinde Naade . Dette Middel er ikke for en oerbar , dydig Mand , som har nok i sig <b>selv</b> ; ikke for en dydig , sat <b>Kone</b> , som tcenkcr at komme i Himmelen , fordi hun opforer sig godt , men det er for de Arme , for Syndere , som ikke kunne hjcelpe sig <b>selv</b> , hvis egen Roes , egen Kraft og egen Dyd er forbi , som ikke lcengere finde nogen Trost i sig <b>selv</b> , og som torste ester Naade , ester Barmhjertighet » . Fur Saadanne er det , for Alle udeu Undtagelse , enten de ere Christne , Isder eller Hedninger ! Alle de , som troe paa den Korsfestede , stulle ikke foiiabes , men have det evige Liv , I ørkenen blev Kobberjlangen opreist paa en hoi Stang , paa det alle Israeliter kunde see den , om de ogsaa vare bidte i den fjerneste Deel af hele Lmen . Saalcdetz er ogsaa Jesus Christus ophsiet for Alles Binc ; Alles Bine skulle ste ham . I de fjerneste Dele af Jorden stal det blive viist den arme Menneskehed, hvorledes Christus hang paa Korsets Trce for Alle stm det reneste Forsoningsoffer ; det stal prcedikes over den ganske Verden , forkyndes med Herolders Slemme , paa det at alle Mennesker i Troen maae stue op til denne Korsscestede , som til deres blodige Borgen og Forlsser ; Hovedindholdet af enhver Prediken af det guddommelige Ord stal verre : Jesus , den Korsfestede ! — Ak , hvor snskede jeg ikke , at jeg havde den Naade , at fremstille Christi Kors saaledes , at det blev seet af Alles Dine !  leg stammer mig ikke ved Christi Evangelium " — siger Apostelen Paulus —  om dei end er denne Verdens Vise til Spot , loderne en Forargelse og Grcekerne en Daarlighed ^ vi prcedike alligevel den korsfcestede Chiistmn , Guds Krast og Guds Viisdom " ( 1 Kor . 1 , 20 — 24 ) . Derfor siger han :  leg agter mig ikke at vide Noget iblandt Eder , uden lesum Christum og ham korsfestet , " derfor siger han til Galaterne ( 3 , 1 ) , at han havde malet Christus for deres Bine , som om han var korsfcestet iblandt dem . Dette skulde vare Hovedindholdet af alle hans apostoliske Prcedikener , for at

<h4>2968</h4>Eder Liggendefe i Himmelen , hvor hverken Mol , ei heller Rust forderver , og hvor Tyve ikke gjennembryde og stjcele . Men soger forst Guds Rige og hans Retferdighed , sta stulle og alle disse Ting tillcegges Eder . " Hermed henviser han os til det himmelske Sind , som vi stulle lade os skjenke , grundlcegges og befestes i os formedelst Guds Aand og Naade . Men hvilket er da dette himmelske Sind , om hvilket vi tale ? — Dersom En soger Guds Rige og hans Retferdighed , dersom En arbeider paa sin egen Saliggjorelse med Frygt og Bceven , dersom En griber og fastholder det ene Fornodne , dersom En kan sige med Apostelen P au lus :  Et gjsr jeg , forglemmende , hvad der er bagved , og rekkende ester det , som er foran , iler jeg mod Maalet til det Klenodie , som horer til Guds Kald herovenfta i Chrifto Jesu " ( Phil . 3 , 14 ) ; dersom En , ligesom Enoch , vandrer med Gud , og har sit Borgerskab i Himmelen , og ikke lenger ssger nogen Glcede i denne Verdens Ting , men higer ester det Uforgjengelige og Evige : — da har han et himmelskt Sind , da opfylder han Budet i Dagens Evangelium , da ligner han den Mand , som byggede sit Huus paa en Klippe , hvor det trodser hver Skylregn og hvert Vandlob og alle Storme . O , et saadant himmelskt Sind er noget Stort , Saligt og Sjeldent ! — At agte det for Intet , som man seer , og at scette Priis paa og elske det , som man ikke seer ; at hcenge ved Gud i Christo formedelst Troen og den rctskasne <b>Kjerlighed</b> og hverken ved Satans Blendverk , eller ved Verdens Forfsrelser og Tillokkelser, eller ved sit eget fordervede <b>Hjertes</b> og Kjsds Drister og Fristelser , — > ikke ved Noget af Alt dette eller ved nogen Ting i Verden at lade sig afoende fra en alvorlig , fast Gang mod Evigheden; at stue hen med ufravendt <b>Blik</b> paa det foreliggende Klenodie, stedse at komme lesum Chiistum , den Korsfestede og Gjenopstandne, ihu , og om end Veien gaaer gjennem mange Gjenvordigheder, gjennem mange Fornegtelser , at esterfolge Lammet , der har elsket os , der har kaldet os , der har opoffret sig for os , — dette er at have et himmelskt Sind , dette er noget overmaade Stort og beroer heller ikke paa nogerr menneskelig Magt ; dette er noget Sjeldent , thi Verden , den store Hob , veed og aner Intet deraf . Og dog er denne Vei i stadig Sogen ester Guds Rige den eneste Vei , paa hvilken vor Sjcel sinder Fred , <b>sand</b> Sjelefred. Thi Gud har lagt Gvigheden i Menneskets <b>Hjerte</b> , og Sjelens dybeste Lengsel kan Intet stille uden Gud alene , den i Chnsto Jesu aabenbarede Gud , som Verden ikke kjender , men ester hvem dog ethvert Menneskehjerte har et hemmeligt Suk og en sterk Lengsel i sit Inderste , om det endog er skjult for Mennesket <b>selv</b> , om Mennesket endog er ligesom nedsunket i Syndens Dynd . O ,

<h4>3071</h4>til at blive rene af Hjertet , som elske oss soge Freden , som da ogsaa Allesammen lide Forfslgelse for Retferdigheds og for Jesu Skyld og lide allehaande Uret — kun disse ere paa den trange Vei , som forer hen til Livet . Men hvo , som ikke soger nogen bedre Retferdighed end de Eknftkloges og Phariseernes Retfer < dighed , hvo , som ikke elsker sin Broder , hvilken , han seer , hvo , som ikke er velvillig mod sin Modstander , medens han er med ham paa Veien , hvo . som bedriver Hoer , om endog kun med Dinene og ikke stnix i sit <b>Hjertes</b> Dyb bliver forferdet over et saadanr <b>Blik</b> , hvo , som ikke overalt soger <b>Sandheden</b> , men endnu er istand til at lyve , til at bande , til at forsverge sig , hvo , som har Sind til at gjengjelde sin Nceste Ondt med Ondt " og ikke langt heller lider Uret , hvo , som endnu kan hade sine Fiender , og ikke streber at elske dem , og ikke beder for dem , — alle Saadanne ere For- » dervelsens Born . Guds Born soge ikke ndvoitcs Anseelse ved sine Gjerninger ; Guds Bsrn sege forst Guds Rige og hans Reiferdighed, de have ikke et <b>Hjerte</b> , som er deelt mellem Gud og Mammon ; de sette sin Fortrostning til den levende Gud ; de kaste sine Sorger paa ham ; de ere altid beskjeftigede med at drage Bjelken ud af sit eget <b>Oie</b> , og derester ste de til at uddrage Skjeden af sin Brodcrs ; de er , e forsigtige i sit Forhold til Verden , de have Bonnens Aand og lade i alle Ting sine Regjeringer ftemfores for Herren i Paakaldelse og Bon ( Phil . 4 , 6 ) ; de soge at elske sin Nask som sig <b>selv</b> . Se , det er den trange Vei , som forer til Livet ! Nu kan Enhver herefter piove sig og see , om han befinder sig paa denne trange Vei . Det har kostet mig stor Overvindelse at troe og predike , at denne er den trange Vei ; jeg vilde heller have ssgt og sundet den paa et andet Sted . Hvor gjerne vilde jeg ikke gjore Veien bredere og mageligere for Eder og for mig ! Hvor gjerne vilde jeg ikke sige til Eder og til mig <b>selv</b> : Kjere Venner ! vi behsve ikke at domme os saa strengt , Frelseren tåger det ikke saa nsie ! Mcn jeg tsr ikke og kan ikke ; det staaer engang saaledes i Guds hellige Ord , og ved dette Ord vil det blive , saa vist som Frelseren har sagt :  Indtil Himmel og Jord forgaae , skal ikke end een den mindste Bogstav eller cen Tsddet foigaae af Loven , ssrend de Ting skee allesammen " ( Match . 5 , 18 ) . Hvad er her at gjore , mine kjere Tilhsrere ! Alle onste vi at blive salige . Ikke <b>sandt</b> ? I Mend , I <b>Kvinder</b> , I Ugifte , I Letstndige ? Er det ikke <b>sandt</b> , I onske dog ogsaa at blive salige ? Ikke <b>sandt</b> ? I Gjerrige ! Ikke <b>sandt</b> ? I Hocrkarle og Skjorlevnere ! Ikke <b>sandt</b> ? I Bedragere , som finde den storste Glede i at kunne tåge en Skilling mere fra eders Neste , end han er skyldig ! I voldsomme , hovmodige , jordisktsindede Mennesker ! Ikke <b>sandt</b> , kjere Tilhorere og Medskyldige for Gud ! Vi onfke dog Alle at blive

<h4>3636</h4>Men Intet ydmyger mere end <b>Sandheden</b> om Christo , naar denne bliver forklaret i Hjertet . Naar han . som vidfte , at Faderen havde givet ham alle Ting iHamderne , reiser sig op og begynder at toe sine Disciples Fsdder ; naar han doer paa Korset under Forhaanelse og Spot som en i Verdens Vine cerelss Misdceder ; naar han , der alene fortjener LEren , bliver saa foragtet « g ringeagtet, at man skjuler sit Anstgt for ham : — hvor skrekkelige , unaturlige , hvilken Vederstyggelighed for Guds <b>Oine</b> maa da ikke disse elendige Indbildninger vere , som Syndere , der ere Stsv , nere i sit <b>Hjerte</b> om sig <b>selv</b> . Men den , for hvem <b>Sandheden</b> om Jesu bliver forklaret i Hjertet ved den Helligaand , han kan kun forbande alle Egenkjaulighedens Lsgne og Indbildninger , alle Stolthedens Tanker . Og kommer nu hertil ydmygende Omstcendigheder, Idmygelstns Veie , paa hvilke Gud ikke lader det mangle , saa bliver Hjertet tilsidst bsiet i Stevet , isandbed fornedret , smeltet til Grunden , og just derfor modtageligt for den evige Herlighed , som Christus vil skjcenke Sine . Man kan saaledes kalde hele Omvendelsens Vceik Menneflets Indforelse i <b>Sandheden</b> . Vi ere af Naturen saa hovmodige , at vi betragte vore usle Regnepenge som Guldstykker . Ved utallige Erfaringer og Idmygelsei lerer Mennestet tilstdst , at de formeentlige Guldstykker ikkun ere usle Regnepenge . Og naar han inoseer dette , da staaer han i <b>Sandheden</b> og i Mmygheden . Men dette er hos Mange en Viisdom , hvortil de fmst bringes efter mange Aars Forlsb . Vi ere engang opfyldte af EgenkjHrlighed , og i denne Egenkjoerlwhed ere vi fsr tilbsielige til at holde <b>Sandhedens</b> Aands Aabenbarelser for Spilfegterier , fsrend vi opgive vore hote og gode Tanker om os <b>selv</b> . Derfor kan vel Mangen overtale sig til at troe , at Guds Aand lader ham see hans Slethed og Forkastelighed saa stor alene for at ydmyge ham ; i Virkeligheden er han derimod ikke saa slet . Han siger derfor : Jeg fsler mig i ncervcerende Bieblik saa utaknemmelig mod Gud , saa forkastelig, saa trceg til at bede , saa lunken og kold i <b>Kjcerligheden</b> til Frelseren o < s . v. ; men , kjaere Menneske ! Du stal dristigt sige : Jeg er ; ikke : Jeg fsler mig . Du er det ; thi den Guds Aand , der revser Dig , er <b>Sandhedens</b> Aand . O , lader os kun mcerke paa Guds Ords og Aands Rost , saa ville vi sikkert tilsidst blive smaa i vore egne <b>Oine</b> og sande , og hvad Satan har udrettet i os , det maa vige bort . Thi med en <b>sand</b> Chriften gaaer det bestandig , som man siger , Krebsgang ; han voxer og skyder Rsdder nedad ; han erkjender for hver Dag tydeligere sin Svaghed og fin Synd , hliver daglig fattigere i Aanden . Dette er nu visselig en Vei , som ikke behager det stolte <b>Hjerte</b> , et bittert Legemiddel for Kjedet , men ssdt for Aanden ;

<h4>3814</h4>begyndt at oprinde , og et klart Skin af den nye Pagtes Dag at trcenge ind i Hjertet ( 2 Petr . 1 , 19 ) . Men hvor dette er steet , hvor Guds Kjaerlighed og Chrifti Uppersteprcestembede bliver aabenbaret i Hjertet , der bortkaster man villig sin egen Netlcerdigheds Kjortel og overgiver sig villig og ganste til den evige <b>Kjcerlighed</b> . Jeg har engang laeft en Fortolling om en beromt engelsk Prcest , som han har fortalt om sig <b>selv</b> ; jeg vil ogsaa fortaelle Eder den . Han sad for Evangeliets Skyld i Famgsel og gjorde her fslgende Erfaring . Han befrygtede , at man omsider fra Fcengselet vilde fore ham til Galgen . Derover kom han i stor Nod , thi der var en Torhed i hans Indre paa den Tid ; de guddom » melige Ting vare skjulte for hans <b>Oine</b> ; han kunde ikke fslenogen Tro hos sig . Der var to Ting , som isar plagede ham , forst Frygten for at han vilde komme til at gjsre Evangeliet Skam med sin Dodsangst , som han ikke vilde kunne skjule i denne Sindstilstand,  thi jeZ stammede mig " — sagde han —  ved at dse for en Sag , som denne var , med Dodens Bleghed og stjaelvende Kncr . " Det Andet , som indjog ham Skroek , var hans Sjcrls Tilstand ester Doden ; hvor vil Du lande , naar Du doer ? Hvorledes vil det gaae Dig ? Hvilken Forvisning har Du om Himmelen, om Herligheden og de Helliges Arvedeel ? Ofte saae han fig i Aanden at fores ud , saae sig staaende paa Stigen med Strikken om Halsen , og hver Gang bcevede hans <b>Hjerte</b> for , at han saaledes stulde gaae ind i Evigheden uden <b>sand</b> Trost , uden levende Haab , maastee med en Angest , der stulde blive til Skam for Evangeliet. Men efterat han i flere Uger havde vceret i denne Anfcegtelse og i dette Morke , kastede han sig endelig ind i den frie Naade . Han erkjendte nemlig , at det kommer an paa Guds Villie , om han vilde stjcenke ham Treft , nu eller i Dsdens Stund ; men at det ikke stod til ham , Mennesket , om han vilde holde fast ved Evangeliet eller ikke ; han var forbunden dertil , men Gud var fri . Derfor tog han nu Mod til sig og sagde ved sig <b>selv</b> : Da Sagen nu staaer saaledes , saa vil jeg gaae ftemad , og vove det paa Grund af min evige Frelse i Christo , enten han saa fortener mig Trsst eller ikke . Vil Gud ikke vaere med , saa lober jeg iblinde fra Stigen og ind i Evigheden , enten jeg synker eller flyder , enten jeg kommer i Himmelen eller i Helvede . Herre Jesu ! vil Du gribe mig , faa gjor det , jeg vover det i dit Navn ! Med denne Tanke veeg Anfcrgtelsen , og Evangeliets Lys og Trost vendte atter tilbage i hans Sjoel . Betragter dog , kj < rre Tilhorere , det vaeldige Spring , som denne M and gjorde ud af alle de i hans Indre kjaempende Tanker og ind i Jesu frie <b>Kjcerligheds</b> Arme . Hvad gjorde da Frelseren, for at bevcege ham til dette Spring ? Han afstar ham

<h4>4613</h4>aabenbares for fin Guds <b>Die</b> : men et ssdt Ord for et Guds Barn , som man ikke kjender i denne Verden , som man miskjender i denne Verden , ligesom ogsaa Frelseren var miskjendt i den , som er iblandt Fremmede og ikke bliver sorstaaet , fordi man ikke sorftaaer Kanaans Tungemaal . Herren siger :  leg veed dine Gjerninger . " Kjcere Tilhsrere ! Om vi end ikke i vore Tider ste en saadan Forfslgelse for Evangeliets Skyld bryde frem , saaledes som det var scedvanligt i den forste Tid , saa kan man dog ogsaa i vore Dage mdnu komme i Stillinger , som i det mindste have nogen Lighed med hine Forfslgelser . Man fslge kun aldeles og uden Hensyn sin Samvittighed i alle Stykker , man vise kun engang et rigtigt Alvor i sin Vandel ; man stmbe kun efter at leve Frelseren til Velbehagelighed , uden at ftygte eller give ester for Mennesker , man verre kun engang en Herrens Werfslger i Gjerning og <b>Sandhed</b> ; man vidne kun engang om Lyset , og lade det stinne ind i Msrket ; man bevise sig engang som en voeldig Jesu Christi Stridsmand , der kaider Sort for sort og Hvidt for hvidt ; — o , da vil det snart vise sig , at dette er Verden en Forargelse , man vil snart finde , at Mennesker , hvilke man har anseet som Venner , forarge sig over en saadan Handlemaade , tale derom , snart spotte og snart beklage Dig ; man vil snart faae at ste , at det til denne Dag er <b>Sandhed</b> , hvad Frelseren har sagt :  Faderen stal voere splidagtig mod Ssnnen , og Ssnnen imod Faderen ; Moderen imod Datteren , og Datteren imod Moderen ; Mandens <b>Moder</b> imod hendes Sons <b>Hustru</b> , og Ssnnens <b>Hustru</b> imod hendes Mands <b>Moder</b> ; thi jeg er ikke kommen at sende Fred , men Svcerd . " la , det kan endog boende , at <b>selv</b> de , som bekjende sig til Frelseren , som regne sig til Zisn , have nok at udscette paa din Handlemaade , saasnart Du begynder , ganske og aldeles at tåge Hensyn til Frelserens Villie , og i Gjerning og <b>Sandhed</b> tåge hans Kors paa Dig , Vistnok have ogsaa Mange skuffet sig i denne Henseende , og have skjult sit Kjsds Villie bag sin Pukken paa Guds Villie , og naar de bleve vifte tilrette af retstaffne Sjoele , friskt voek anvendt disse Ord paa sig <b>selv</b> :  Vil mig <b>selv</b> da Zion domme ? " — og have derhos fulgt sine Skjsdesynder , og have skjult sine Lyfter og Be- Mringer under et christeligt Klcedebon ; thi det lader sig ikke udfige, hvor stor Selvstuffelsen kan voere ; — men jeg taler om saadanne Mennesker , som ere guddommelig ) forvisstde om sin Sag og redelige , som ikke ville gjere sin egen Villie , men Faderens Villie , og i fit <b>Hjertes</b> indeiste Grund ere sorsikkrede derom . Saadanne stulle kun engang begynde at vende sit Ansigt lige mod det himmelske Jerusalem , hvortil aldeles ikke udfordres nogle overordentlige Gjerninger , de stulle kun i sit daglige Liv tragte ester ikke lomgere at stikke sig lige med Verden i nogen Ting , men ved

<h4>4643</h4>et , forlader de nioghalvfemsindstyve i Brkenen , og gaacr bort ester det tabte , indtil han sinder det ; dette er hans <b>Hjertes</b> vigtigste Anliggende . Men en anden , ligesaa stor Omsorg af den gode Hyrde er den , at den , som engang er grebet af ham , ikke paa nogen som helst Maade stal blive reven ud af hans almcegtige Hyrdehaand , men at hans Faar maa blive bevarede fra det Onde indtil hans Tilkommelses Dag . Hvor inderlig har ikke den store Uppersteprceft i fin sidste Bon anbefalet sine Disciple og sine Troende til Faderen : „ Jeg beder ikke , at Du stal tåge dem ud af Verden , men at Du skal bevare dem fra det Onde . De ere ikke as Verden , ligesom jeg er ikke af Verden . Hellige Fader ! Hellige dem i <b>Sandhed</b> ; dit Ord er <b>Sandhed</b> " ( Joh . 17 , 15 — 17 ) . O , hvor udgaae ikke disse Ord fra den trofaste Ippersteprasts <b>Hjertes</b> inderste , dybeste Grund , han , som med uendelig <b>Kjcerlighed</b> omslutter og omfatter Sine , og hvor fremvcelder ikke deraf det varme Bnste at see Sine urokkelig ! befcestede i <b>Sandheden</b> og bevarede i hans Haand , uden at rives ud af den ! Ligesom hans <b>Hjerte</b> stog og bramdte af <b>Kjcerlighed</b> til hans Disciple , saaledes bramder det endnu lige til vore Tider for alle hans Troende , ja endog for den ringeste af hans Brodre . Derfor luede ogsaa i Apostlernes Hjerter en saa hellig Flamme af den inderligste <b>Kjcerlighed</b> til Brsdrene ; <b>Blikket</b> i Jesu kjcerlighedsfulde <b>Hjerte</b> havde antcendt den hos dem . Derfor loese vi ide apostoliske Breve , at Paulus stedse i sine Bonner tcenker paa Menighederne , at de skulde mcegtigen bekrceftes i Henseende til det indvortes Menneske , rodfcestes og grundftestes i <b>Kjoerlighed</b> , at de skulde . opvoxe til ham , som er Hovedet , til Christus ; derfor advarer han dem som en Fader og Broder fra Affald , fra Utroskab , fra Vantro , fra Forfsrere og Vranglærere ; derfor stuver han til Ebrceerne :  Seer til , Brodre ! at der ikke i Nogen af Eder stal vcere et ondt , vantro <b>Hjerte</b> , saa at han affalder fta den levende Gud " ( Ebr . 3 , 12 ) . Det Gamme laa ogsaa Apostelen Johannes paa Hjertet, da han strev til sine Lcesere :  Mine Born ! bliver i ham , at naar han aabenbares , vi da kunne bave Frimodighed , og ikke stulle blive bestjcemmede af ham i hans Tilkommelse " ( 1 Joh . 2 , 28 ) . Det var Apostelens <b>Hjertes</b> inderligste Lcengsel , at de Sjcele , som engang va ^ e qrebne af Jesus Christo , ogsaa maatte holde trofaste ud indtil hans Tilkommeise . Men hvorfra havde de denne inderlige <b>Kjcerlighed</b> , uden fra Herren <b>selv</b> ? O , kjoere Tilhsrere ! Frelserens <b>Hjerte</b> banker af <b>Kjcerlighed</b> til hans Troende ; han har hengivet sit Liv i Doden for dem , han har udsst sit Blod for dem , <b>Kjcrrligheden</b> har drevet ham ind i den dybeste Dsdsnsd , har gjort ham til et for dem ; derfor havde han heller intet hsiere Onste , end at frelse Sine , at gjere Sine evig salige , og

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015090808025'target=_>, 1876, Sandheden i Kristus</a><h4>54</h4>FM den 16 de Oktober 1631 i Lubeck af fromme borgerlige Forceldre , modtog han en omhyggelig Opdragelse og lcerte tidlig af sin <b>Moder</b> at bede med Alvor og Andagt . Udrustet med store Aandsgaver, kunde han allerede i sit 14 de Aar besoge Hoistolen i Rostock og siden fuldende sine Studier saa hurtigt , at han allerede i sit nittende Aar begav sig paa Reiser , og , vendt tilbage igjen til Rostock , 21 Aar gammel , modtog en LErke-Diakons Embede . Aaret derefter blev han Doktor i Theologien og det folgende Aar tillige Profesfor . Ikke lcenge efter sin Ansættelse cegtede han den fromme Elisabeth Siebrand , Datter af en agtet Kjobmand i Rostock . Om sin for ste Embedsvirksomhed ytrer han sig <b>selv</b> saaledes : „ Da jeg tiltraadte det hoihellige Embede , som jeg nu forer i Aandens Kraft , var jeg angst og bange ; thi jeg var endnu uerfaren og havde uovedeSandser i guddommelige Ting og lidet Mod til at straffe de Ugudelige . Hvad skulde jeg gjore ? ' For min Gud nedkncelede jeg i mit Kammer og sagde med Jeremias : Ak , Herre , Herre ! se , jeg ved ikke at tale , thi jeg er ung . Men Herren sagde til mig : „ Sig ikke : jeg er ung , thi du stal gaa til alt det , som jeg sender dig til , og alt det , som jeg befaler dig , stal du tale . Frygt ikke deres Ansigt , thi jeg er med dig at redde dig , siger Herren . " At dette Ord er et <b>sandt</b> Ord , har jeg erfaret . Dem Gud sender , salver han ; sine Sendebud gjor han duelige og udruster dem med Kreefter til at fore sit Embede . " Hau vandt snart sin Menigheds <b>Kjcerlighed</b> og Tillid og sogte ligeoverfor Bagvaskere Trost i sin gode Samvittighed . Paa de Studerende virkede han til stor Velsignelse , idet han ikke blot sogte at danne deres Aand , men ogsaa deres Hjerter . Med trofast Forbon bar han dem ham betroede Sjcele paa sit <b>Hjerte</b> . Flere Kaldelser til andre Hoistoler og store Etceder afslog han . Han folte sig bunden til sin Rosenstok , saa kaldte han Rostock . „ Jeg har her , siger han , en god Menighed , der agter mig som en Guds Engel , annammer til Opbyggelse Herrens Ord , prcediket ved mig , og overoser mig med

<h4>592</h4>Betragter jeg ellers vor Teft og betcenker , hvem der vel pcm det ømmeste har elsket vor Herre Jesus , saa var det udentvivl haus <b>Moder</b> Marias Mme <b>Hjerte</b> . O hvad maa der have gaaet for sig ! hvilke Kjcerligheds-Flammer maa her ikke have vceret til dette Barn , som huu havde ftdt , og om hvilket der var sagt hende saa store Ting ! Altsaa sige vi da heller ikke uden Grund , at <b>Kjcerligheden</b> til deu Herre Jesus er allerMmest hos en saadan Sjcel , der aandeligvis er bleven Kristi <b>Moder</b> , saaledes og i den Forstand , som Kristus <b>selv</b> taler derom ( Matth . 13 , 49. 50 ) . Men for Kortheds Skyld ville vi for denne Gang ikke tale mere derom , helst siden det ogsaa vil blive klart af det Følgende . II . Vi have nu for det Audet ogsaa at betragte : Hvorledes denne <b>Kjcrrlighed</b> til den Herre Jesus viser sig . Om Josef og Maria hedder det i Testen : De forundrede sig over de Ting , som bleve sagte om ham . Se , der viser sig den ftrste <b>Kjcerlighed</b> til den Herre lesns i denne Fornndring . Der gives vel mangestags Forundring , idet den kan saavel komme af Bcmknndighed og Vantro , som af en af Gud virket Erkjendelse og en <b>sand</b> Tro . Men hos Josef og Maria er det aabenbart , at deres Forundring var en Frugt af deres Tro og var forbunden med en stor Glcede over alt det Gode , som var dem sagt om den Herre Jesus . Nu maatte de vel kalde dette Barn ret underligt , ligesom Esaias havde anvist Kap . 9 , 6. Enhver maa <b>selv</b> videre eftertcenke denne Sag , hvilket visselig , om det ster ret , ikke vil gaa af uden Frugt .

<h4>1160</h4>hvad Virkning har dette havt hos dig ? Har dn vel efter den Anvisning , du har faaet , retteligen omvendt dig til din Skaber og GjenlFser ? Har du vel ladet Ordet bevise sin Kraft paa dig til et nyt Liv , til en <b>sand</b> <b>Hjertes</b> Forandring , saa at din oprigtige Attråa nu ftaar til at leve et benaadet og ret saligt Liv i Jesu <b>Kjerlighed</b> ? Kan du ikke besvare disse SpFrgsmaal med et frimodigt Ja , saa sig mig , hvor skulde da vel det Ord , du i saa mange Aar har hsrt , vcere blevet af , dersom ikke Sjelefienden havde bortftjaalet det efterhaanden , altsom du har HM det ? Jo , kjere Sjel . ' saa er det i <b>Sandhed</b> . Tror du , hvad Frelseren <b>selv</b> siger , at Djevelen har taget Ordet af dit <b>Hjerte</b> . Og hvorfor har han gjort dette ? Paa det at dn ikke skulde tro . Naar det siges dig , at du endnn er uomveudt , at dn gaar borte fra din Frelser , i Vantro og Synd og Elendighed / saa vil Satan ikke , at du stal tro dette , derfor tager han strar saadanne Ord bort , at de ikke skulle treuge ind i dit <b>Hjerte</b> . Naar det siges dig , at Guds SM brender af en faa usigelig <b>Kjerlighed</b> til dig , at han gjerne vilde endnu i dette Dieblik forlofe dig sra al Syndens Magt og gjsre dig fuldkommen falig , saa vil Satan ikke , at du stal tro dette ; thi han ser vel , at , dersom du ret troede det , saa vilde du strar gaa hen til dcunc kjerlige Ven og lade ham hjelpe dig , men din Sjels Fiende under dig ikke at blive frelst , hau gjor Alt , hvad han kan , for at lesn kjMe Syndere ikke skulle tro og blive salige . 2 ) Den anden Hindring for Ordets Virkning er det menneskelige <b>Hjertes</b> Haarohed . Frelseren ligner det ved en Klippe og det fortjener fandelig at lignes derved . Man maa ofte med ligefaa megen Forundring som Sorg se , hvor haardt et Menneskes <b>Hjerte</b> kan vere . Naar Jesu <b>Kjerlighed</b> til Menneskene anprises paa det Allerhoieste , naar hans for dem udstandne Marter og Lidelse afmales med de mest levende Farver , naar deres Synde-Elendighed og Sjele-Ve paa det Allertydeligste stilles dem for <b>Oie</b> , naar de paa det AllerMmeste advares og bedes om at soge deres evige Frelse , at omvende sig til den Herre , som ikke vil nogen Synders Dpd , og som derfor er ganet faa vidt i sin <b>Kjerlighed</b> , at han har kjM dem med sit Blod , naar Alt dette siges dem saa Mt og rerende , at man ofte kunde ville hensmelte i Taarer,

<h4>3424</h4>ham i vort <b>Hjerte</b> med sin levende Griffel . Han forklarer Frelferm af hele hans Historie , fom er bestredet i Ordet , hos dem , der stan i inderlig Omgang med ham . Han bringer Frelferens hele Historie saa noer det Gud sigende <b>Hjerte</b> , fom som om den foregik just nn for vor Sjcels <b>Oine</b> . Den ene Gang ser man Frelferen i Krybben , den anden Gang fom Dreng , som Yngling , som Mand ; man ser ham i hele hans hellige Livsvandring her paa Jorden ; man fer ham i hans Angst , Marter og Pine , i hans Saar , i hans i hans Grav , efter hans Opstandelse , og undertiden bliver ogfaa lidt af hans Herlighed aabenbaret for Hjertet . O , hvilket klart , glcedeligt Lys kan ikke ofte et lidet Sprog af den hellige Skrift ved den Hclligaands Kraft udbrede over den deiligste blandt Menneskens hvorledes kan ikke Sjcelen blive optcendt af <b>Kjcerlighed</b> til ham ved et eneste saadant Lysblik ! Hvor vel ftler ikke Hjertet sig ved Nydelsen af slige himmelske Krcefter ! Helst forklarer <b>Sandhedens</b> Aand ham vistnok i den store Forretning , da han lcd og dFde for os fom vor Forfoner og Borgen . Da kan man staa under hans Kors , som om han <b>selv</b> netop hang paa det ; da kan man se hans Blod flyde , som om det netop nu fl > d til Forsoning for vore Synder ; da fer man ind i hans Saar , fom om de netop nu vare slagne ; man ser det , man drikker Naade og Forbarmelse af dette Syn for sit frelfebegjcerlige <b>Hjerte</b> . — O , hvilken Vederkvægelse ! Hvor bliver man ikke draget hen til sin ydmyge , sin fornedrede , sin forflaaede , pinte , forfmcegtende , diende Jesus og Midler , til sin allerdciligste Frelser !

<h4>3699</h4>ning i og under ham med en <b>Kjcerlighed</b> , som os taknemmelige og dobbelt kjcerlige mod ham <b>selv</b> . At disse Tings <b>Kjcerlighed</b> bliver syndig , sier ikke ftrend da , naar den bekæmper, svcekker og hindrer Guds <b>Kjcerlighed</b> . Hvor dette ikke sker , der beholder Gud endnn det ganske <b>Hjerte</b> ; thi han har Herredom i Hjertet , ligesom en Mand siges at bebo og besidde sit ganske Hus , endstjMt hans Hnstrn , VM : , Venner og Tjenere bo ogsaa deri med ham , men under ham og efter hans egen Vilje . Kortsagt , det ganske <b>Hjerte</b> , som udkrceves til Gnds <b>Kjcerlighed</b> , er egentlig den Udelelighed , Oprigtighed og Redelighed , med hvilken man begjcerer Gnd , opofrer sig til Gnd , og det til sin store Hovedsag at tcekkes ham med alle derimod stridende Tings Fornegtelse . Her hedder det : Hvo , som elsker Fader eller <b>Moder</b> , <b>Hustru</b> eller Bsrn , Brodre eller Sostre mere end mig , han er mig ikke voerd . Dette er da Meningen af Lovens fyrste og største Bud om at elske Herren vor Gud af gauste <b>Hjerte</b> , af ganske Sjcel og af ganske Sind . Komme vi nn til en SelvprMelse , da sindes jo Nogle , som aldeles ikke elske , men hade Gud ; Andre , som elske ham tilsyneladende og af et halvt eller delt <b>Hjerte</b> ; men nogle Faa , som elske ham af ganske <b>Hjerte</b> , stjMt endda ikke ncer saa heftigt , som han er vcerd . De Fyrste ere hans aabenbare Fiender , de Andre hans falske Venner , det tredie Slags hans igjensMe VM og sande Hjertens Venner . I Joh . 5 , 42 siger lesns nogle haarde Ord til Ilderne , nemlig: Jeg kjender Eder , at I have ikke Guds <b>Kjoerlighed</b> i Eder . Vi , som ikke kjende hverandres Hjerter , ikke altid fcelde saadan dristig Dom . Dog findes altfor Mange af det Slags Mennesker , som <b>selv</b> kjendelig nok aabenbare og blotte den samme usalige Tilstand , nemlig de , som efter Jesu Ord i Joh . 3 , 20 hade Lyset , det er , Guds <b>Sandhed</b> og altsaa Gud <b>selv</b> . Disse Msrkheds BM , Gnds Fiender og Retfcerdigheds Foragtere vil jeg alene advare , at det vil blive dem tnngt , ja omsider umuligt at stampe mod Braadden . Kan Guds <b>Kjcerlighed</b> ikke overtale dem , da stal omsider hans Vrede fortcrre dem . Sige disse Daarer nu i deres Hjerter : Der er ingen Gud ( Ps . 14 , 1 ) ; da stulle de dog i hin Verden faa Troen i Hcenderne , naar det stal hedde : Mer mig h : d disse mine Fiender , som ikke vilde , jeg stnlde herske over dem ,

<h4>3782</h4>lups Glcede og Festlighed , eller hMr Tale derom , saa oplade du dine <b>Oine</b> og dit <b>Hjerte</b> og fe , hvad din kjcere Herre Kristus deri foreholder og vifer dig . Ogsaa han har , saafremt du tror paa ham , beredt dig som sin kjcere Brud og et levende Lem , et herligt , kongeligt Bryllnp , hvor der er evig Glcede , gode Dage , Syngen og Springen , evig Prydelfe , al Rigdom og alle gode Tings Fylde . Ogfaa derved skal en saadan hjertelig Fortrøstning til ham voxe og tiltage i dig , at han af usigelig hjertelig <b>Kjcerlighed</b> ved Daaben har kaldet og udvalgt dig til dette Samfund og antaget sig dig for at forlise dig fra Syndens , den evige og Satans Magt og derfor ofret sit Legeme og Liv og Alt , hvad han har , paa dig , ja , hengivet sig saa aldeles for dig , at du ikke alene trMgen og gladeligen kan rose dig af det , som han for din Skyld har gjort og sijcenket dig , men ogsaa af ham <b>selv</b> som din egen Eiendom . Og ligesom en Brnd med hjertelig Tillid forlader sig paa sin Brudgom og holder hans <b>Hjerte</b> for sit eget <b>Hjerte</b> , saaledes bM ogsaa du af Hjertens Grund forlade dig paa Kristi Kcerlighed og vcere forvisset om , at han ikke er anderledes sindet imod dig end som dit eget <b>Hjerte</b> . Men herimod strider saare mcegtigt vor gamle Adams , det er , vort KjM og Blods , Blindhed og Mesl > se Haardhed , som ikke lader os se eller tro dette , iscer naar vi i os <b>selv</b> og i dette elendige Liv se og mcerke noget ganske Andet med vore Dine og Scmdser . Thi det ser og forstaar vel ogsaa Fornuften, at et Bryllup og Brudekjcerlighed i sig <b>selv</b> er et lifligt og lysteligt Billede ; den indrMmer vel ogsaa , at Kristus er en stjM , cedel , from og trofast Brudgom og hans Kirke en herlig og salig Brud . Men naar Enhver for sig stal tro , at ogsaa han hFrer Kristus til og er et Lem paa dette Legeme , og at Kristus bcerer saadan hjertelig <b>Kjcerlighed</b> til ham , da vil Fornuften ikke lcenger Mge med. Det kommer deraf , at jeg ikke ser en saadan ypperlig Herlighed hos mig , men tvertimod ser og ftler stor Svaghed og Uvcerdighed , idel Bedrøvelse og Tungsindighed , ja og Grav og Orme , som stulle fortcere mig . Men derimod stal du lcere at tro det Ord , som Kristus <b>selv</b> siger dig , og som Gud befaler dig at tro . Det stal du holde for <b>Sandhed</b> , hvis du ikke vil gjFre ham til en Lagner

<h4>4270</h4>Diseiple , som daglig hMe hans Ord , blive jo lamger , jo mere indtagne , overbeviste og fast knyttede til ham ved hans Ord . Det samme som afholdt Farisæernes Tjenere fra at gribe Jesus , det samme afholdt hans Disciple fra at forlade ham : Hvor stulle vi gaa hen o . s . v. Mine Allerkjcereste ! Hvor stnlle vi gaa hen i denne Fastetid? Hvor stulle vi forlyste vore Sjcele ? Hvor stulle vi med stirre Andagt og Lyst ftge vor Opbyggelse end i lesns og hans Ord ? Han har det evige Livs Ord . Hans Ord og hans Blod ere vel de dyrebareste Ting , vi knnne betragte , den dyrebareste Skat , vor Sjcel kan forlange . Da vi tilforn have betragtet adskillige Stykker af Jesu blodige Lidelse , saa ville vi i dette Aar , efter den mig dertil givne Anledning , forlyste os med Jesu Ord , de Ord , Livet talte i Dydens Stund . De syv Ord , Jesus talte paa Korset , de vare ret Livets Drd , da han er Livet <b>selv</b> . Jeg er Veien , <b>Sandheden</b> og Livet , siger han ( Joh . 14 , 6 ) . Han er Livets Fyrste , ja et diende Liv . Han , Livet <b>selv</b> , som nu dFde for at gjFre os leveude , talte disse Livets Ord , ftr han Se vi hen til Materien og Indholdet af Ordene , da er der intet Andet end Liv og Naade , <b>Kjcerlighed</b> og Salighed i hvert et Ord . Der er <b>Kjcerlighed</b> til hans Fjender ; thi han beder : Fader , forlad dem ! Thi de vide ikke , hvad degjMe ! <b>Kjcerlighed</b> til hans Venner ; <b>Kvinde</b> , siger han , se , det er din SM , og til Johannes : Se , det er din <b>Moder</b> ! <b>Kjcerlighed</b> til Syndere ; thi han siger til RMeren : Idag stal du vcere med mig iParadis ! <b>Kjcerlighed</b> til hans Fader ; thi han raabte : Min Gud , miu Gud , hvorfor har du forladt mig ? og : Fader , jeg befaler min Aand i dine Hcender ! Scmledes er hvert Ord idel <b>Kjcerligheds</b> Ord , Livets Ord . Der ere saa mange Aarsager , hvorfor disse Ord vcere os fremfor alle Jesu Ord dyrebare og kjcere og med scerdeles Andagt betragtes . De ere komne af hans Mund , der bM vcere os kjcerere end alle Munde i Verden , af hans Mund , i hvilken aldrig blev funden Svig , aldrig HM et unyttigt Ord . Dette kunne vi ogsaa sige om alle hans Ord , men iscer om disse ; thi de ere en diende Mands Ord , og de Ord , som et diende Menneske taler i sin Dødsstund , Pleier man dog at lcegge baade vel paa <b>Hjerte</b> og tillige at bevare lcenge i Minde , saa vi ikke lettelig glemme dem , efterdi vi vide , at

<h4>4461</h4>at faa det nye Navn skreven paa Stenen , som Kristus lover i Aab . 2 , 17 , uden han har havt dette Navn paa Jorden . Men ville vi have dette Navn , saa lcere vi af Johannes , hvorledes vi kunne faa det . Det kan ikke hos nogen øvrighed, ikke kjKes for Penge , ikke faaes ved Anbefalinger , men Vil du elskes , saa clst ; vil du vcere en Discipel , som Jesus elsker , vcere en Discipel , som elsker Jesus , saa hold hans Bud ; thi derpaa kjendes det , at vi elste ham , om vi holde hans Bud ( 1 Joh . 5 , 3 ) . Gn Trsft er her ogsaa i disse Ord for Enker og Forladte, meu den TrLt strcekker sig ogsaa til alle Guds efterdi ikke alene Maria er Jesu <b>Moder</b> , men alle de , som gMe hans Faders Vilje , hvilket han <b>selv</b> siger . Ikke alene Johannes er hans Discipel , men alle de , som fornegte sig <b>selv</b> og tage sit Kors paa og fKge ham . Saa angaar ogsaa denne Forsorg Alle . Han ved vel , at Ingen maa lide mer i Verden end hans <b>Moder</b> og Disciple , hans og Lemmer : han ved vel , at der er Ingen , som vil sMge for dem . Verden sarger for sine Vare I af Verden , saa elskede Verden Eder ( Joh . 15 , 19 ) . Velan da , saa vil Jesus <b>selv</b> for dem . Den , som her viste sin forladte <b>Moder</b> en Johannes til Truster og Støttestav , han vil og efter sin Visdom og <b>Kjcerlighcd</b> i Nodens Tid opvcekke Middel , Raad og Venner, hvis <b>Hjerte</b> han <b>selv</b> vil oplade . Det er hans Jeg vil ikke slaa Haanden af dig ( Hebr . 13 , 5. Derfor kan en troende Kristen frit overlade ' sig til den trofaste Frelser, og smukt dybt uedsceuke sig i dette hans Ord paa Korset . Han har endnu et <b>kjcerligt</b> <b>Die</b> at se med ; som da af sit Kors , saa nu af sin Himmel ser han paa dem , som opløfte Dine og <b>Hjerte</b> til ham . Han har endnu et medlidende <b>Hjerte</b> , saa han ynkes over vor han har endnu en venlig Tnnge , hvormed han truster os ; han har endnu eu eller anden Ven , som han kan formåa til at antage sig os . Vel den , fom tror paa ham , vel den , som ikke gaar til de falske BrMde , som ei kunne holde Vand ( Jer . 2 , 13 ) , men holder sig til ham , den levende Kilde . Hans Forsorg er vedvarende ; som han havde elsket Sine , saa elskede han dem indtil Gndcn ( Joh . 13 , 1 ) . Vi maa derfor sige om ham , som Naomi om Boas : Den Mand hviler ikke , fpr han idag gjpr Ende paa denne Sag ( Rut 3 , 18 ) .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011060704053'target=_>Essendrop, C.P.P., 1873, Prædikener til alle Aarets Søn- og Helligdage</a><h4>558</h4>Hvor Herrens Aand er , der er <b>Sandhed</b> , Vilje til at se . Evne til at kjende <b>Sandheden</b> . ~ I Herrens Lys skulle vi se Lys , " medens det naturlige Menneske ikke kan dmnme aandelig . Hvad se vi i HEgteskabet ? Maaske kun et Forhold , der tiltaler vor Sands ved sin Behagelighed under dette Livs Forhalde , en Overenskomst , som har sit Vcerd ved at sikre os et Hjem med Pleie og Deltagelse i Hjemmet , en Samfundsordning , der har sin umistelige Betydning for Familie og Stat under disses timelige Forhold , en Forbindelse , som tjener til den gjensidige Mfylden af Mandens og <b>Kvindens</b> Vcesen og deres Udvikling for det Liv , som nu er , et Kjcerlighedsforhold , som tilfredsstiller vor naturlige Trang , styrker vori Arbejde for denne Berden og bygger vor Fremtid i vor Efterstcegt . Vi stulle ikke her npiere veie Rigtigheden af en saadan Betragtning af LEgteskabet , ei heller svsrge , om den , som kun ssgte Saadant i 2 Egtestab , ogsaa virkelig fandt , hvad han sogte , og fandt sig tilfredsstillet derved . Men det vide vi , at hvad <b>Sandhed</b> der kan vcere i saadan Betragtning af LEgteskabet , det er doa . ikke den hele <b>Sandhed</b> af , hvad ZEgtestabet skulde vcere ester Guds Tanke og Plan . Kommer den Herre Jesus ind i LEgtefcellers Hjerter og Hus , vidner Herrens <b>Sandhed</b> Andet og Mere for dem . Han giver Sine et rigere og dybere Syn paa , hvad Wgtestabet er , hvad det stal vcere ester hans hellige Vilje , som indstiftede det . Det er bestandigt ved Herrens <b>Sandhed</b> , det hsrer med til <b>Sandhedens</b> Virksomhrd i Frelsens Tjeneste , det hprer saaledes med til den Herres Jesu Aabenbarelse af sin Frelserherlighed , at han lcerer os at se enhver Sag i dens Forhold til den levende Gud og et evigt Liv i ham . Til en christen Betragtning af 3 Egtestabet tMer det fprst og fremst deri at se en Guds Indstiftelse , som henter sin Berettigelse , sit Indhold og sin Bestemmelse fra Guds hellige Vilje . Herrens Egne ville iog med sit LEgtestab det , som deres Gud og Herre vil med 3 Egtestabet , og nu tpr de jo vistnok med fuld Tillid tro og hjertelig glcede sig i , at Gud har <b>Hjerte</b> og Tanke ogsaa for deres timelige Liv i alle dets Forholde , at han gjerne vil glcede og styrke dem ogsaa i Naturens Rige , gjerne vil tilfredsstille enhver Trang , som han <b>selv</b> har nedlagt i sine Skabningers Natur , gjerne ser mennestelig <b>Kjcerlighed</b> at

<h4>590</h4>sig fta sin Ondskab . Og som Gud Herren havde givet dem det faste , urokkelige Lofte , at sende just dem en Frelser , saa havde han ogsaa forjoettet dem i ham Liv og evig Salighed , havde saa sikkert lovet dem , at han ikke vilde forstode nogen af dem , som annammede hans Udsendte , at , vare deres Synder end rodere end Skarlagenets Farve , skulde de dog ved Frelserens Forsoning blive hvide , som den snehvioe Ulo . Men han havde ogsaa tilfoiet saa mangt et Varfelsord . Som det var idel Naade , at de havde faaet nogen Velsignelse , nogen Forjcrttelse , saa skulde det og ste , at hver den Sjcrl , som ikke monne hore den Profet , skulde udslettes af Folket , skulde ikke lcengere have Arv og Lod med Herrens udvalgte Eiendomsfolk , hans det sande Israel . Men Abrahams forvildede Born overhørte altfor ofte det Sidste , forvandlede de betingede Lofter til übetingede Forsikringer og holdt <b>selv</b> kun fast ved de übetingede Forjoettelser , forsaavidt de tiltalte deres kjodelige Sands . Det Ord , som var givet dem til Opreisning , blev dem til Fald ; hvao der i deres <b>Hjerte</b> skulde have foot Idmyghed og <b>Kjoerlighed</b> , brugte deres forvendte Sind til Ncering for Hovmod og Ukjcerlighed . <b>Selv</b> dromte de sig sikre paa Guds Rige , hvordanne de saa end vare , om end deres Hcender vare fulde af Blod , deres Hjerter langt fta Herren , og Hedningerne , de skulde vcere aldeles udelukkede ; saa dybt som Hedningerne stode under Israels Born , skulde Naadens Time aldrig oprinde over dem . la , der vare nok ikke saa Faa blandt de forvildede loder , hos hvem den Mening ret syntes at fode Glcrde , at de alene skulde arve Guds Rige , at de Andre stet ikke skulde faa Lod og Del den . Derfor faldt de ogsaa for <b>Sandheden</b> . Da Israels Haab kom til sine Egne , fuld af Naade og <b>Sandhed</b> , kom med oybeste og rigeste Ovfyldelse af Forjcettelserne til Fcedrene , annammede hans Egne ham ikke . Han var ikke saadan , som de havde ventet ham ; han talte ikke de Ord , som de havde Lyst til at hore . De annammede ham ikke ; de hadede og forfulgte ham og naglede ham omsider til Korsets Troe . Og <b>selv</b> de , som vare af <b>Sandheden</b> , de , som dog solte sig dragne hen til ham , hans <b>Sandhed</b> og hans <b>Kjcerlighed</b> , som fornam , at han havde det evige Livs Ord , — hvor svag og uklar var dog deres Erkjendelfe , hvor lidet dybt og omfattende deres <b>Blik</b> , hvor lcenge vareoe det ikke , inden det bestemt og kraftigt lod : ~ Du er Christus , den levende Guds Son . " Og

<h4>858</h4>Vende vi nu <b>Blikket</b> til os <b>selv</b> , MedforlMe , og svsrge , hvad Dagens Evangelium har at sige os , da maa vi vel , saasandt <b>Sandhed</b> har Magt over vort <b>Hjerte</b> , fsrst og fremst Me os dybt bestjcemmede ligeoverfor <b>Kvinden</b> fra Kanaan . Allerede det / at hun ssgte hen til Jesus , har Meget at sige os . Det var saa Lidet , hun havde hprt af Sandhedsordet , medens hun visselig paa den anden Side havde he » rt saa Meget , som kunde have fsrt hende bort fra al <b>Sandhed</b> , som maatte gjsre hende det saa saare vansteligt at fatte og tilegne sig <b>Sandheden</b> . Men , hvor lidet det end var , havde hun dog bevaret det , og det havde hun ikke gjort , dersom ikke hendes <b>Hjerte</b> havds lcert at torste efter <b>Sandheden</b> . Vi have HM . saa Meget , saa saars Meget , have endog he » rt det store Ord om Korsets Forsoning , det Ord , som just i denne Fastens Tid dobbelt skulde daglig staa for vor Sjcrl , over os ftraaler i det profetiske og apostoliske Ord <b>Sandhedens</b> fulde , klare Lys ; men med hvor stor Trofiab have vi bevaret det ? Hvor dybt og inderligt have vi optaget det i os , ladet det ret virke paa os , saa vi fulgte Ordet og gik til ham , den Herre Jesus Christus , vor Herre og Frelser , om hvem hvert Sandhedsord vidner , gik til ham og overgav os ganste og aldeles til ham , ventende alt Godt af hans milde og moegtige Haand ? Vel sanot , det var maaske ftrft Sorgens tunge Tryk , der bragte <b>Kvinden</b> til ret at grunde paa , hvad hun havds HM , det var maaske fM i Npdens Time , i Smertens bitre Stund , at Kvindsn tcenkte paa Jesus , tcenkte paa at ssge Hjcelp og Redning hos ham . Men gaar det saa med os Alle ? Vi have vel mere end een Gang , Enhver paa sin Maade , erfaret Nsd og Sorg og Smerte ; men saa ofte gaar det dog saa , at Nsden og Smerten istedetfor at drage Hjertet til den treenige Gud , til hans <b>Kjoerlighed</b> og rige Barmhjertighed , drager os bort fra ham . Vi fordybe os i vor Sorg , se kun , vil kun se denne , indtil vi forhcerde os mod den eller fortvivle i den . Og hvordan fan end vort Liv fowvrigt var , om overstwmmende rigt paa alle dette Livs Goder , aldeles frit for det , som i Almindelighed kaldes Sorger , een Sorg have vi dog Alle at berre paa , Sorgen over vor Synd og vore Synder . Havde vi ret agtet <b>Sandhedens</b> Overbevisning , maatte vi dog forlomgst have ftlt denne Sorgs

<h4>864</h4>og , hvor forstjelligt han end fsrer de Forstjellige , gjennem Kamp maa de dog Alle gaa , gjennem en saameget haardere og wngere Kamp , jo Mere der endnu er hos dem af deres eget Vcesen , jo mindre Christus endnu er bleven Alt i dem . Det er lifligere end Duggen at bo paa Herrens hellige Bjerg , at hpre den Herre Jesus M , og Sandhedsordet skuffer ikke , naar det forjcetter dem , som i <b>Sandhed</b> hpre Herren til , hans Fred . Men Fredens Hjem er i det himmelske Jerusalem , i Faderens evige Boliger , og , giver end Herren af sin Naade Sine allerede herneden de Timer , da det b liv er saa stille og lunt og godt inden i dem , og hans Fred mildt overskygger dem , saa er det dog vist ogsaa saa , at just Herrens Egne ret faa Me den svare Trcengsel , tet maa gaa gjennem den haarde Kamp . <b>Selv</b> den almindelige Sorg og Modgang , som den kaldes , som mere eller mindre rammer alle Swvets Bprn , <b>selv</b> den bliver i visse Maader noget langt Mere , langt Tungere for den Troende end for det naturlige Menneske . Hin ser dybere og veed , hvad det er , som er Sorgens <b>Moder</b> , fsler Syndens Braad i Alt , hvad der er tungt . Men M veeo han ogsaa , at det er Frelserens <b>Kjcerlighed</b> , som er ogsna i dette , at det kun er hans Egenvilje , som stal brydes , hans eget Vcesen , som stal udryddes , at Herrens Hjoelv er rede og skal meddeles os da , naar Troen gjennem Prpven og Kampen har naaet den Inderlighed , den Renhed og Styrke , at vi , ret til Fred for os , kunne smage Herrens Naade . Den , som i Troen veed Hg greben af Herren , skal i Troen vcere sikker paa , at Frelseren forer ham gjennem Merket til Lyset , gjennem Trcengsel til Fred , gjennem Dpd til Liv , og med dette Lys over sine mprke Stier kan han ikke frygte dem , med dette Haab kan han ikke forsage . Jesu <b>Kjcerlighed</b> er hans eneste , men saa ogsaa hans rige Trpst , og derfor er han saa inderlig fattig , naar denne unddrager sig ham . Derfor er det hans tungeste Timer , naar det vil tykkes ham , at Frelseren gaar ham kold forbi uden Vie for hans uden Ore for hans Raab . Det er hans haardeste Prßve ogsaa i en saadan Stund at holde Ordet fast om den <b>Kjcerligheds</b> evige Nforanderlighed , som synes at unddrage sig ham , at bygge fast i Tro paa den <b>Sandhed</b> : Christus Jesus er min Frelser , da , naar Herren <b>selv</b> synes at vise os , at han ikke vil vcere vor Frelser , og vort <b>Hjerte</b> vidner for os , at Herren kan have altfor god Grund til at lade os falde . Bevarer hans

<h4>1009</h4>Og Haabets profetiske Forjcettelse er for os bleven den lyslevende , mangfaldig omvidnede og bekrceftede Ovfyldelse ; thi Guds Son er virkelig bleven fodt af en <b>Kvinde</b> formedelst den Helligaand . Ordet er blevet Kje » d , ja kom i syndigt Kjods Lignelse , men ved sin hellige Unofangelse uden Synd og var bestandig i Lydighed indtil ja Korsets Dgd . Det er ikke Haabets og Haabets Glcedes Evangelium , som Dagens hellige Tekst forkynder os , men Ovfyldelsens , den virkelige Ovfyldelses gode og glade Evangelium . Hans Navn er kaldet Jesus , alt efter hvad Engelen bebudede Maria ; han har frelst sit Folk fra deres Synder ; han har nu besteget Davids , sin Faders , Throne og er en Konge over lakobs Hus evindelig ; han udsender fra sin Himmelthrone sin Aand i alle Hjerter , som ville annamme ham , og gaar i den i Troen annammeoe Helligaand <b>selv</b> ind i en Synders <b>Hjerte</b> , og med ham Guds Fred og Guds <b>Kjcerlighed</b> . Saa opvcelder derinde i det for saa torre <b>Hjerte</b> en Kilde til det evige Liv , og dermed vcelder Glcedeskilden frem , og det samme Menneske faar Aandens Naade og Aandens Kraft til at glcedes i sin evige Glcede ved de mange Gud har skabt for ham i Naturens , i Menneskesamfundets , i den hellige Aands Verden . la , til Christi Forlpste heder det : ~ Glcel » er Eder , atter siger jeg , glceder Eder i Herren ! " Christenlivet treenger som Menneskelivet til Glceden , og i Christus er Glceden ret berettiget , dyrekjpbt som den er os ved Perlerne af Christi Blod . Det er saa inderlig vel begrundet , at den , som endnu kun har feet <b>Sandheden</b> i Dommen over hans Synd , gaar med det urolige <b>Hjerte</b> , det morke <b>Blik</b> , den sorgbetyngede

<h4>1047</h4>at det var din Synd , som istjcmkede ham den bitre Kalk , din Synd , som flog ham til Forbandelsens Troe . Og naar nu Jesu <b>Kjcerlighed</b> er falden paa dit <b>Hjerte</b> , naar hvert et Trin , han gaar , moner om hans Frelsestanker over dig , stiller da ikke det Spprgsmaal sig frem for din Sjcel : Hvorledes har jeg linnet min Frelsers <b>Kjcerlighed</b> ? Har jeg ofret mig til ham , som han har ofret sig for mig ? Jo mere <b>Kjcerligheden</b> , som hans Aand har stabt i dit <b>Hjerte</b> , driver dig til at cere ham , som dyrekjMe dig til sin Eiendom , desto mere maa dog ogsaa Aanden bedwves inden i dig , naar du ser , hvor fattig din <b>Kjcerlighed</b> til ham har vceret mod hans <b>Kjcerlighed</b> til dig , hvor halvt dit Offer for ham har vceret mod hans Offer for dig . Jo mere din Tanke i denne Uge ganske hendrages til Frelseren , jo mere dit <b>Hjerte</b> fslger ham paa Korsgangen , desto mere ville ogfaa Minderne om din megen Utrostab stige frem i din Sjcel , desto mere ville disse Minder fylde din Sjcel med Bedwvelse . Fordi Aanden har lcert dig at have <b>Hjerte</b> for den Wd , hvorfra Frelseren ene har udfriet dig , og for Frelserens store <b>Kjcerlighed</b> og store Gjerning , derfor vil det ogfaa vederfares dig , hvad der vederfores Frelferen under Indtoget: ~ Der han saa Staden , grced han over den og sagde : O , at du dog paa denne Dag vidfte , hvad der tjener til din Fred ! O , at du dog kjendte din Beftgelses Tid ! Men nu er det skjult for dine Bine . " Der ere Vidnesbyrd , som raabe til Himlen , at der ogsaa mellem os ere Mange , Mange , som ikke kjende sin Bessgelses Tid , Mange , som fornegte den Herre , forn kjpbte dem , og stulle fsre over sig <b>selv</b> en hastig Fordcervelse . Tgrhcende ere der i Kredsen as dine Egne , af dem , som <b>selv</b> ved Naturens Baand ere tcet knyttede til dit <b>Hjerte</b> , de , om hvem du , naar du ikke vitterlig vil lukke dine Bine eller lyve mod <b>Sandheden</b> , maa vide , at de ikke have det evige Liv , thi de have ikke den korsfcestede Jesus i sit <b>Hjerte</b> . Da maa det dog dobbelt i denne llge , da Alt raaber , at der kun er Frelse i Christus Jesus , at der er mtet Andet givet os til Salighed end ham korsfcestet , falde dig bittert tungt paa Hjertet , at disse saaledes stulle krcenke Herrens <b>Kjcerlighed</b> og fornegte hans ZSre , at de stulle saaledes forkaste de dyre Gåver , forn bydes dem , faaledes stcenge sig <b>selv</b> ude fra Naaden og Livet , og Ugen bliver dig atter en fort Sorgens Uge . Dit <b>Blik</b> vil igjen spge fra disfe hen til din kjcere Herre og Mester og Frelser , men du tager Mindet med dig af , hvad du saa og

<h4>1560</h4>Men , ere vi i Christus , saa har hans Himmelfart og Scrde ved Faderens Hpire dog ikke alene det Gode for os , at der forsikres os et Hjem i de mange Vcerelser i Faderens Hus deroppe, og den Magt over os , at Lcengselen vcekkes ester at fare hjem til vor Gud og Fader . Ere vi i Christus , saa er ogsaa vort <b>Hjerte</b> allerede nu bundet til ham . Som vi i Troen have ham , griber ikke alene Lcengselen ester , hvad han har gjemt til os , og hvad han engang vil give os ; men Troen og <b>Kjcerligheden</b> griber , hvad han allerede nu vil stjcenke , hvad han allerede nu giver dem , som ham annamme . ~ Ere I ovreiste med Christus , da soger det , som er oventil , hvor Christus sidder hos Guds hoire Haand ! Tragter efter det , som er oventil , ikke efter det , som er paa lorden ! " Den christne Formaning.til det christne <b>Hjerte</b> <b>moder</b> jo vistnok villig Opfattelse og villig Efterfolgelse , fordi den samme Aand , som er i den christne Formaning , ogsaa er i det christne <b>Hjerte</b> ; men dog treenger denne Aand til at nceres , som jo ogsaa den Troenoe bcerer Aandens Skatte i et skrobeligt Lerkar , og det naturlige Vcesen i ham ligefrem staar Aanden imod . Og det , som ncerer Aanden , er just Troens <b>Blik</b> paa Frelseren . Troen ihukommer ham , saadan , som han er . Herren til Gud Faders 3 Ere med det hellige Sind , det rene <b>Hjerte</b> , med hver Begjcering rettet mod det , som er as <b>Kjcerlighed</b> og <b>Sandhed</b> . Herrens Tanke er aldrig fcengslet til det Jordiske , hans <b>Hjerte</b> aldrig draget af , hvad der er Daarlighed og Forfcengelighed , aldrig af , hvad der er Gud imod . Hvad Skatte han har vundet for os , hvad Gåver han har at give Sine , kunne ikke forliges med et verdsligt Sind , et forfcengeligt <b>Hjerte</b> eller en af Synden trcelbunden Aand . Troens <b>Blik</b> ser hen til Frelseren , til hans Ihukommelse , og , ihukommet i Troen , drager Frelseren vaany og vel ogsaa stedse inderligere og stcerkere paa Hjertet . Men ikke det aleneste . Troen tager og oser af Frelserens Vceld , Troen indsuger Frelserens Liv ; den Troende forvandles alt mere og mere ester sin Frelsers hellige Lignelse , soger alt mere og mere de Ting , som ere heroventil , hvor Christus er , siooende hos Faderens Hoire . Frelserens

<h4>1568</h4>maa og stal lcegge sig tungt over Guds Vsrns Hjerter . Synden er dog en Vcemmelse for Aanden , som den er en stammelig og formastelig Kromkelse af Guds hellige 2 Ere . Vel <b>sandt</b> , en cegte Christen kan ikke se Synden hos Medvandrerne og dsmme den , uden at <b>Sandhedens</b> Aand tvinger hans <b>Blik</b> indad til at se og kjende Syndens Mcerker i hans eget <b>Hjerte</b> og hans eget Liv . Det Syn kommer vel ogsaa , om han saa stod ganske ene og Intet mcrrkede til Synden i Verden udenfor ham . Men , fordi han finder Synden ogsaa hos sig <b>selv</b> , derfor bliver den vel saa langtfra mindre smertelig for hans aandelige Sands , at der kun fsier sig en ny Smerte til , at Bitterheden bliver dobbelt . Fremdeles , en Christen elster sin Herre , elster virkelig og oprigtig, og <b>Kjcerligheden</b> glceder sig , naar den Kjceres Herlighed anerkjendes og elskes , <b>Kjcerligheden</b> sorger , naar den Kjcere fornegtes og krcenkes . Der falde mange Smerter vaa Guds Bglrns Vei , naar de saa daglig maa ovleve det , at de Mange gaa Christus koldt forbi , at de Mange fornegte ham , at der ere de , som håane ham og hans Sag . Det brcender jo i Spnnens <b>Hjerte</b> , naar <b>Moder</b> spottes ; bitter Harme tamdes i Vennens <b>Hjerte</b> , naar han maa hore Vennens 3 Ere krcenket . Men saadan Smerte faa Guds Born ret ofte gjennemgaa ikke alene fra dem , som med bevidft Sind og beraad Hv staa Herrens Salvede imod , men ogsaa fra dem , som mene , at de have Del i Christus , skjM de aldrig have kjendt ham , . saadan , som han er , og , hvad vcerre er , ogsaa i sine egne Hjerter finde Guds Born , ogsaa fra sine egne Hjerter se de det ovstige , som er Herrens hellige <b>Kjcerlighed</b> imod . Der er en dyb , ofte fljcerende Smerte deri .

<h4>1641</h4>fpdtes i Jesu hellige Navn , er det ftemdeles , saasandt der her hjemme og derude endnu ved Aandens Virksomhed lcegges Sjcele til Menigheden . Men ikke det aleneste . Den Helligaand forlader ikke sin egen Skabning . Igjen berpvet Aanden , var Kirken ikke lcengere et Guds Hus og en Guds Familie , saa vist , som da heller ikke den Sjcel er et Lem af Jesu Christi Kirke , som engang havde Troen , men kun den , som har Troen , Aandens Tro og Troens Aand . I ikke , at I ere Guds Tempel , og at Guds Aand bor i Eder , " det er <b>Sandhedens</b> ogLErens Ord om Guds levende Menighed . Aanden iet Hus , den rorer sig jo ikke atene inden Huset , men den lever i Hjerterne inde i Huset ; saa ogsaa med den Helligaand som Jesu Christi Kirkes Aand . Ser du Herrens Kirte som Staden , der straaler Lys ud i Verdens mprke Nat ? Det er Aandens Herlighed , som straaler i Lyset og er Lyset . Hprer du Idrcetternes Lyd af , hvad der fier derinde ? Det er Aandens Vcesen og Aandens Kraft , aabenbarer sig derudi . la , hvert enkelt af dette Huses Lemmer , som du maatte mode paa din Vei , dig for det , thi det samme Mennefie bcerer Guds hellige Aand i sit <b>Hjerte</b> ! I enhver enkelt Jesu Christi Troende som i det hele de Helliges Samfund er den Helligaand Lyset og Livet og Kraften , er deres Aand , og det ikke alene glimtvis i de enkelte he » ie Timer , da <b>selv</b> Verden studsende og forundret fornemmer , at her er noget Stort og Overordentligt paafcerde , og maafke i grundig Hjerteraahed frcekt vil vcerge sig mod Indtrykket paa samme Maade som hine , der idag sagde om Herrens Avoftle , at de vare fulde af spd Vin , men ogsaa i de Me og lave Timer , ja endog ide tunge Stunder , da Stjernens Straaler ganske trcekke sig ind , og kun det Nllgne Korstegn bliver tilbage . Lyset er kun i Gud , som det er af Gud ; Livet er kun Liv , faalcenge det altid paany ftrpmmer ud af Guds evige Livskilde , og Kraften kan aldrig fijcenkes et Menneskebarn anderledes , end at den i hvert Bieblik paany meddeles af ham , som ene eier al Kraft . Og Herren , vor Gud med den evige Frelsens Tanke og Frelsens Attraa over vore Sjcele , glemmer ikke og forlader ikke Sine , om saa en <b>Moder</b> kunde glemme sit Livs Barn . For vor Gud med hans uendelige <b>Kjcerligheds</b> Attraa er det en Lyst at faa bo og leve i et Mennestes <b>Hjerte</b> og . faa gjensveite sit Villede i dets rensede Dyb . Hans Israel er hans <b>Hjertes</b> Brud , med hvem og i hvem han begjcerer

<h4>1844</h4>Herren udlcegger <b>selv</b> denne <b>Sandhed</b> i de to Lignelser i vor Dags hellige Tekst og foier dertil , som I alle vide , Lignelsen om den fortabte og gjenfundne Son . Hyrden , som soger ester det bortkomne Faar , <b>Kvinden</b> , som leder efter den tabte Penge , det er vor Herre og Frelser , som dyrt og fuldt har forhvervet sig Navnet af den gode Hyrde , som med al Frelserens Troskab , al <b>Kjcerlighedens</b> Inderlighed soger efter det Fortabte , og Faderen er vor Herre Jesu Christi Fader , som , Eet med sin enbaarne SM , Eet med ham ogsaa i Frelsens Vilje og Attraa , i glad Faderkjcerlighed modtager hver den Sjcel , som omvender sig til ham . Det bortkomne Faar , den tabte Penge , den forlorne SM , det er dig og mig . Ven ! O , at Gud i fuld Forstand kunde sige til os : Det er Eder ! Bortkommen , tabt , forloren , det er Ord , som med sit morke Indhold passe altfor godt vaa hvert et Menneskebarn , som er fodt af syndig Sced . Engang var Mennesket af Guds Art og Slcegt med Guds Tanker i sin Sjcel , Guds Vilje i sin Begjcering , Guds <b>Kjcerlighed</b> i sit <b>Hjerte</b> ; engang var hvert Befog af Gud en Fredshilsen , et saligt Mode af Fader og Barn . Var den hellige Fortcelling herom kun et Sagn , ja endog kun et Billede , fodt i en Digters Sjcel , maatte det dog tale mildt til vort <b>Hjerte</b> og vcekke de dybeste Lcengfter detinde , og da saa meget mere , naar vi vide og tro , at Fortcellingen ikke er Sagn , end sige Digt , men den rene og klare <b>Sandhed</b> , aabenbaret for os og beseglet af Gud <b>selv</b> . Hvad Menneskelivet da maa have vceret ! Hvad Fryd det maa have vceret at trcede vaa Guds Jord , at se ind i Guds Vie , at lcegge sit Ord i Guds Pre , at tåge Guds Ord af Guds Mund ! Vel maa Lcengsten stige efter at blive , hvad vi ikke ere , efter at vinde igjen , hvab der er tabt . Du kan jo ikke hengive dig til hin Betragtning , du kan ikke fordybe dig i Synet af Paradisets Herlighed , noen at der kommer Graad i dit <b>Hjerte</b> , om ikke Taarer i dit Bie , fordi du veed og kjender , at du

<h4>2555</h4>og levendegjorde Manasse . Eller , om man beder , stiger dog Bemnen kun mat og uden Haab ; der er ikke ißsnnen den Troens Kraft , som vinder Israels-Seieren . Vi komme ikke vor Broder tilhjcelp med vor Tro , <b>selv</b> om vi ville komme ham tilhjcelp med vor <b>Kjcerlighed</b> , og derfor kunne vi heller ikke overvinde alle Hindringer for vor Fremtrcengen til Jesus med vor syge Broder . Der er dyb <b>Sandhed</b> i det Svar , hin <b>Moder</b> gav sin Sem , der var en Ordets Tjener mellem sine Brsdre , da han klagede over , at hans Vidnesbyrd var forgjceves , at Gudsordet i hans Mund prellede tilbage fra de kolde Hjerter : Men tror du da ogsaa ret , at dit Vidnesbyrd stal bcere Frugt paa Hjerterne ? Tror du det ikke , hvorledes kan du da vente And et , end at Frugten stal blive derefter ? Har du din Broder kjcer , og vil du arbeide paa at omvende ham fra hans Veis Vildfarelser , saa gaa i Troen for ham hen til vor Herre Jesus Christus ! Se ikke ene hen til din Broders Msrke , men ogsaa og det ret inderligt og bestemt til Lyset og Kraften og <b>Kjcerligheden</b> i Herren ! Som du er dybt nedbsiet over hins Elendighed , saa glced dig i den samme Stund over din Frelsers store og underlige Magt , og hold dig ret fast til , at Herren stod over , den Vcerkbrudne, som var lagt ved hans Floder , og sagde fyrst : ~ SM , vcer frimodig ! Dine Synder ere dig forladte , " og saa : ~ Staa op og gaa ! " Naar du beder Troens Bpn for dig felv og for din Broder , saa veed du dog , at din Bon ret egentlig er streven ud af Herrens <b>Hjerte</b> , at han <b>selv</b> lcegger sit Ia og Amen dertil , og Herren stal i sin Naade gjsre ikke alene , som du troede , men ogsaa i mange Maader gjsre det ved din Tro .

<h4>2662</h4>som ydmyger , som tvinger til at flåa Viet ned . Vi kjende vel alle den Folelse af Skamfuldhed , som overvcelder os , naar En , for hvem vi have nogen Agtelse , griber os i at gjore det , som ikke fommer os ; men hvad stulle vi sige , naar vi kjende , at Herren , vor Gud , sek > , han med den hoie Majestcrt , han med den fulde Hellighed , er kommen over os og har grebet os i vor Synd og vor Skam , og det ikke alene i en enkelt Synd , som vi kunde forsoge at bilde os ind , havde overlistet os , men i vor hele syndige Tilstand . Maa ikke den Son , som Faderen , forend han sendte ham ud i Verden , rigelig udrustede med Alt , hvad han paa sin Vandring kunde tiltrcrnge , men s « m nu igjen staar for Faderens Dor nogen og elendig , forbi han i Syndens Tjeneste har forsvildt alle Faderens Gåver , maa han ikke blues og stamme sig dybt og inderligt over at staa saaledes for Faderens Ansigt ? Og vi staa for Ens Vie , som er meget Mere end en jordisk Fader , som havde udrustet os med noget langt Mere og Bedre end med Klceder for Legemet , med Gods og med Guld . Vi burde jo havt megen Ros for vor Gud ; men hvad er vor Ros ? Den er borte , og i dens Sted tilkommer os vore Ansigters Blusel . Og vi kunne ikke sige , at Faderen glemte Sonnen i det fremmede Land ; visselig , nu ialfald , naar vi have faaet Vinene op , fe vi det , at Gud ikke har ladet os uden Advarsel , ikke har undladt atter og atter at kalde os bort fra de vildfarende Veie , som det lystede os at betrcede , bort fra den Fordcervelse , som vort daarlige <b>Hjerte</b> valgte istedetfor at tjene de. n levende Gud , og dog varede det saa lcenge , inden vi paa nogen Maade Horte Ordet og boiede os under det , og , da vi omsider Horte det , var det dog saa lidet med vor ganske Sjcrl og vort ganske <b>Hjerte</b> . Vi maa vel klage og grcede over os <b>selv</b> , og den Guds <b>Kjcerlighed</b> , som har baaret os til denne Dag med en Barmhjertighed og Langmodighed , som ingen Mennefietunge rettelig kan bestrive , maa dobbelt fylde os med Skam og og Blusel , dobbelt staa vort Bie ned . Og det er jo visselig <b>sandt</b> , det Ord , at Synd fortjener Straf , at det er en forfcrrdelig Ting at falde i den levende Guds Haand . Den , som med <b>Sandhed</b> i Hjertet ser sin Synd , begjcerer inderlig Syndernes Forladelse , hungrer og torster ester Retfcrrdighed . Han staar med Tolderen for Gud med nedstaaet <b>Blik</b> og beder i Oprigtighed : ~ Gud , vcer mig Synder naadig ! " ,

<h4>2795</h4>isinefaloende nok , at hun ikke havde den samme stcerke Tro paa hans Viljes altforbarmende <b>Kjcerlighed</b> . Tcenkte hun saa paa Mere ? Tcenkte hun paa Jesus som Messias , som den af Gud lovede Frelser ? Begjcrrede hun i den samme Stund Mere end Helbredelse fra Legemets Sygdom ? Begjcerede hun sin Sjcels Frelse fra Syndens Npd ? Vi vide det ikke ; ialfalo var dette endnu ikke klart i hendes Sjcrl , var endnu ikke blevet hende det Fyrste ; thi hvor det kommer op til Klarhed og Bestemthed i Sjcelen og ikke forkastes af Hjertet , der bliver det det Fprste for Sjcelen , det Stprste for Hjertet . Men Herren tcenkte det over hende og begjcrrede det for hende , og for dette Maal vilde han derfor ogsaa fsrft ikke alene opfylde hendes Attraa efter Helbredelse , men ogsaa give hende den levende Overbevisning , at han med fuld og kjcerlia . Vilje vilde lade fin Kraft vorde hende til Del . Derfor giver han hende fprst af sin Livsfylde til Helbredelfe , den Meddelelse af hans Kraft , som <b>Kvinden</b> i Troen paa hans Kraft nu kunde modtage og modtog , og npder hende saa til Bekjendelse for at kunne svare hende : ~ D atter , vcer frimodig ! Din Tro har frelft dig ! " Og hun forstod , hvor villig hans Vilje var til at hjcelpe , forn hans Magt havde vist sig stor ; hun fornam Noget af hans <b>Kjcerligheds</b> Rigdom , og hans Gjerning og hans Ord og hans <b>Blik</b> drog hendes <b>Hjerte</b> til ham , og du vil forståa , hvorledes deri var baade Bpielse og Opreisning , og vi haabe for vist , at om det ikke straks kom til det fulde Gjennembrud , til fuld Klarhed og Bestemthed , saa var der dog nu lagt det i hendes <b>Hjerte</b> , som blev til den Bedrsvelse efter Gud , som virker Omvendelse til Salighed , og faa til den Tro , som griber den frelsende Naade i Christus Jesus og- af ham tager og tilegner sig Forsoningens Gave og Forlpsningens Krcefter , og da , Medforloste , blev det i end fuldere Maal <b>sandt</b> over hende , at hendes Tro frelste hende .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013010208009'target=_>Müller, Heinrich, 1860, Himmelsk Kjærlighedskys</a><h4>1664</h4>som David beder i den 51 de Psalme : Gud , skab i mig et reent <b>Hjerte</b> og fornye en stadig Aand i mit Inderstes ) . Hvad en Mester har gjort med sine Hcender , det behager ham . Det Menneske er Christi Dannelse , som Paulus stuver : Indtil Christus faaer sin Dannelse i Eder 2 ) . Nu er Christus Grunden til alt Guds Velbehag : Denne er min elskelige Ssn , i hvilken jeg har raaber Faderen <b>selv</b> fra Himmelen . Det nye Menneske er Guds Billede , som Paulus skriver om det ; Det er stabt efter Gud i <b>Sandhedens</b> Retfærdighed og Hellighed ^ ) . Ligheden er en <b>Moder</b> til <b>Kjcerlighed</b> , Kærligheden til Velbehag . Joseph behagede sin Fader i den brogede Kjortel , som han <b>selv</b> havde givet ham . Gud beklcedcr os med Christo ; thi saa mange som ere dsbte paa Christum , de have ifsrt sig Christum ; naar vi da ere ifsrte denne Prydelse , saa forlyster Gud sig i os ; hans eget Barn , hans egen Prydelse maa jo behage ham . Hos Propheten Esaias finde vi disse trsstelige Ord : Ikke lcrngere stal du kaldes : den Forstudie , og dit Land ikke loengere : det Dde . Nei , du stal hedde : den af mig Elstte ; og dit Land : Mgtehustruen ; thi Herren vil elste dig , og dit Land stal forenes med sin Mgtefoelle . Thi som Inglingen cegter en Ms , saa vil din Opbygger cegte dig ; og som Brudgommen gloeder sig ved sin Brud , saa stal din Gud gloede sig ved dig ^ ) . Jeg tor sige , at jeg ei i den ganske Skrift har lcest dejligere Ord om Gud , end disse , i hvilke han er os afmalet som en <b>kjcerlig</b> Ven og Brudgom , der ikke veed at beholde sin <b>Kjcerlighed</b> , men udsser den over sin Brud i Hobeviis i hendes Skjod , kysser og kjcertegner hende , ja veed neppe for indvortes Lyst og Glcede , hvorledes han stal tee sig . Forestil dig en saadan Ven og Brudgom , saa har du et Billede paa , hvorledes Gud tilbyder sig <b>selv</b> til en troende Sjcel , hvilket dog kun er et Billede og en Skygge , altfor ringe til fuldkommen at kunne afmale Guds overvættes rige <b>Kjoerlighed</b> . Ak ! hvor <b>kjcerlig</b> omdsber han sit Barn , saaar den Forladte faaer en Mgtchustru , og Orkenen bliver til en Lysthave .

<h4>2258</h4>noieAgt paa vort <b>Hjertes</b> Tilboieligheder , hvori vi som oftest soge os <b>selv</b> , og deri oftest og kraftigst forncegte os <b>selv</b> . 3 ) Naar vi ofte anstille Betragtninger saaledes , at vi ogsaa tillige i Hjertet smage og fole , hvor oprigtig og hjertelig Gud og Christus have meent det med os . Hiin har ikke sparet sin egen Son , men gav ham hen for os i den viltreste Dod ; denne derimod har saa smertelig ladet sit <b>Hjerte</b> blode for os paa Korset . See ! saa kunne vi blive antcendte til Gjenkjoerlighed , til igjen at tjene ham oprigtig i Rand og <b>Sandhed</b> . Dette mener David , naar han siger : Din <b>Kjcerlighed</b> er mig for <b>Die</b> , og jeg vandrer i din <b>Sandhed</b> ' ) . Det maatte sandelig voere et ugudeligt <b>Hjerte</b> , som ikke stulde mene Gud det trofast , da han dog mener os det saa vel .

<h4>2279</h4>Saa grundig og dyb er denne <b>Kjoerlighed</b> , den vil gjerne meddele Sjoelene sin Godhed , og naar Synderen hindrer det , da sorger den . Dette kan intet <b>Hjerte</b> begribe , at Gud , der i sit Boesen ikke kan siges at lide noget , dog lider i sin Kjoerlighed, ikke for at faae Noget , men for at meddele sig <b>selv</b> , ligesom naar en <b>Moder</b> rcekker sit Barn Brysterne , men Barnet gaacr bort , og foragter dem , da gjor det Moderen saa ondt , at hun groeder . Hvo kan uden Taarer tcenke paa den sode Frelsers hjertelige Medlidenheds- og Kjoerlighedstaarer , som han udgjsd over det jodiste Folks Halstarrighed ? Slige Taarer groeder han endnu den Dag idag , naar vi stode hans Godhed fra os , ja var det muligt , maatte hans hellige Saar derover springe op , og groede Blod . Paulus formaner Epheserne : Bedrover ikke den Helligaand , med hvilken I ere beseglede indtil Forlosningens Dag / ) . Den Helligaand har sin Skole i Hjertet , der loerer , formaner, advarer og troster han . Ligesom nu en from Skolemester bliver bedrovet , naar han seer for sine <b>Oine</b> , at hans Disciple ikke agte hans Formaninger , saaledes bedrsve vi ogsaa den Helligaand , naar vi hoenge ved Synden , som han indvortes straffer i Hjertet .

<h4>2351</h4>tungeste Voegt er , saaledes bsier Guds Barmhjertighed sig derhen , hvor der er megen Elendighed . Og ligesom en <b>Moder</b> har langt stsrre <b>Kjoerlighed</b> til et lidet og grcrdende Barn ; thi de Spcedeste cre altid de Kjoereste , saaledes er Guds Barmhjertighed stsrst imod dem , der ere smaae , ringe og elendige i deres Hjerter . I den 113 de Psalme siges Herren at ste paa Himmel og Jord i ) . Guds <b>Oie</b> betyder hans Kjcrrlighed, ligesom jeg ofte og venlig vender Vinene til den , som jeg elsker ; og ligesom en Fader seer paa sit svage og strsbelige Barn med forbarmende Sine . David siger : Ved din Godhed beredte du det for den Elendiges . Ved Guds Godhed forstaaes Gud <b>selv</b> , guddommeligt Liv , Lys , Gloede , Guds Naade , Syndernes Forladelse , Sjaelens indvortes Fred , Tro , <b>Kjoerlighed</b> , Haab , Andagt og kraftig Trsst . Disse gode Ting kunne ikkun de Elendige blive deelagtige i . De som have den jordiske Trsst i Hjertet , kunne ikke smage den himmelske; derfor er den Elendiges gode Ting langt bedre , end al Verdens jordiske Skatte for den Rige . Skriften kalder saadanne naadelsse Hjerter „ torre Hjerter " . De Elendige og Fattige lede efter Vand , og der er intet , deres Tunge forsmægter af Torste ) . Ligesom man paa et tort Sted ikke finder Vand , saaledes sinder man i et saadant <b>Hjerte</b> hverken Aand eller Trsst .

<h4>2787</h4>sig fra det hellige Bud , som var dem overantvordet . Men det er gaaet dem efter det sande Ordsprog : Hunden vender sig igjen til sit eget Spy , og Soen , som var toet , til flidden Ssle ^ ) . Men Gud vcrre lovet , der befcester de Fromme , og fuldfører den gode Gjerning , som han har begyndt i dem , indtil Jesu Christi Dag . Ligefremme og trostelige Ord ere de , som Paulus skriver til sine Corinthier : Gud stal stadfæste Eder indtil Enden , saa at I skulle vcere ustraffelige paa vor Herres Jesu Christi Dag . Gud er trofast , ved hvem I ere kaldte til hans Ssns Jesu Christi vor Herres Samfunds . Og atter : Gud er trofast , som stal ikke lade Eder fristes over Eders Formue , men stal gjore Udgangen med Fristelsen , at I det kunne Peder stemmer ogsaa overeens heri : Al Naadens Gud , som kaldte os til sin evige Herlighed i Christo Jesu , han <b>selv</b> berede , styrke , bekrcefte , grundfæste Eder ! som lide en kort Tid ^ ) . Mcrrk her din farlige og elendige Tilstand , af dig <b>selv</b> er du som en Fcrstning uden Volde og Mure , som Fienden let kan bestige og overvinde , ja Fienden og Forrideren er midt i Fæstningen , dit eget <b>Hjerte</b> , der er Djcevelen mere tro mod dig , end du <b>selv</b> . Din Tro og Bon er som Muur og Bryst- - vcrrn elendig , sonderreven , Stykkevcerk , Pjalter og Tiggervcerk. Du er som en Leerpotte , der let kan brydes ; som et lidet svagt og sygt Barn , hvilket man kan omstode med en Finger ; som et Huus , der er bygget paa <b>Sand</b> , og ikke er sikker mod nogen Storm . Men mcrrk ogsaa , hvad Guds <b>Kjcrrlighed</b> gjsr ved dig ! Den gjsr Fæstningen fast og uovervindelig , den forbedrer og istandsoetter , hvad der er odelagt paa den . Er Troen svag , da holder Guds <b>Kjcerlighed</b> den ved Haanden , som et lidet Barn , og giver den megen sod Trsstemelk at drikke , saa at den derved tager til , og bliver styrket . Denne <b>Kjcrrlighed</b> bygger Huset paa en Klippe , lcegger sig <b>selv</b> til Grundvold , saa at Verden og Djcevelen forgjceves lsbe Storm derpaa . Det kommer af , at den er en tro <b>Moder</b> ; hvad den har begyndt i Naaden , vil den ogsaa gjerne fuldfsre . Den er en

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015062608033'target=_>Rosenius, C.O., 1879, Hemmeligheder i Lov og Evangelium</a><h4>360</h4>altid var tilbsielig til det onde . og hele Menneskeheden dod i Overtrædelser og Synder , indesluttet i Djcrvelens Lcrnker , belagt med Guds Brede og Forbandelse og laa i Elendighedens Dyb op over Hovedet Vl kunde da billigtvis forundres over , at Herlighedens Herre doa siger : Min Lyft hos Menneslens Born . „ Min Lyft " . Hvad er nu det ? Hvad kan vcrre Guds Lyst ? Sandelig , aller forst det . som fra Evighed var hans Lyst . Fryd og Fornoielse , da der endnu ikke var skabt nogen Ting , som kunde fryde hans Vie . Hvad var da Gjenstand for hans Fornoiette ? Sandelig , kun han <b>selv</b> Kristus . Thi han alene er i sig <b>selv</b> stjon , elskværdig og Guds <b>Hjertes</b> Lyst , og alt andet er kun stjont , for saa vidt som det er en Del af ham eller udflyder af ham . Herren har sin Fornoiclse i sig <b>selv</b> ; thi han <b>selv</b> er den eneste evige Skjonhcd . Kun Straalerne af hans egne Fuldkommenheder tilfredsstille hans <b>Oie</b> . Og denne hans Oies Lyst den skulde vcere hos Menneskens Born ? Ja , hvor skulde den ellers vcere ? Hos Englene maastee ? De have deres egen Herlighed , men Kristi Herlighed Syndere . Ja . hvor underligt det end kan lyde , det er dog <b>Sandhed</b> og det er noget endnu forunderligere : han har afklcedt sig <b>selv</b> det kongelige Purpur og stjcenket det til sine fattige , men igjenloste Vrodre . Fader , siger han , jeg har givet dem den Herlighed , som Du har givet mig — jeg helliger Mig <b>selv</b> for dem ( Joh . 1 ? . ) . „ I kjende jo vor Herres Jesu Kristi Naade . at han for Eders Styld blev fattig , der han bar rig . paa det I ved hans Fattigdom skulde blive rige " ( 2 Kor . 8. 9. ) . Han blev gjort til Synd for os , paa det bi skulle vwre Guds Netfcerdighed i ham ( 2 Kor . 5 , 21. ) . Han har ikke villet beholde sin Retfærdigheds hvide Silke for sig <b>selv</b> , men givet sin Brud det til Bedcekmng og Prydelse , og den . . fattige Orm Jakob " pranger i haus Glands , i hans Juveler . Kjceder og Kroner ( Est . 16 , ) . Al . ja , hvor skulde vi ellers kunne vcere frimodige , om ikke hans Retfærdighed bcdcokkede vore Synder ? Hvor skulde vi nogensinde kunne have Fred med Gud . have en god og glad Samvittighed og omgaaes fortroligt med ham , for hvem itte engang Himlene ere rene , om vl ikte vidste , at vi ere bctlcrdtc med den Fuldkomnes Fuldkommenhed og den eneste Helliges Hellighed ? Hvor skulde vi brcekkcde Ror saa frimodigt kunne trodse Djcevelen og alle hans Hccre . om vi ikke vidste , at vi hade den Almægtiges Venstab ? Hvor skulde vi kunne ovloftc vore Hoveder og med Glcede vente Herrens store Dommedag , om vi ikke vare indsvobte fra Hovedet til Foddcrne i hans glinsende Lydighed , og om vi ikke for Gud kunde trodse paa hans Retfærdighed , paa hans <b>Kjcerlighed</b> , hans Herlighed som vor egen . som paa en os virkelig stjcenket Eiendom ? Her kan visselig nogen anmcerke : „ Det er just ingen ftor Kunst at prange i en andens Fortjeneste og lyse i en andens , Glands ! " Kan saa vcere . og dog . hvem gjor os det efter ? Forfog. hvor let denne Kunst er ! Imidlertid er det nok , at det dog . er Mledes , at hans Herlighed er os given . Vom nys A ? y nnfort ,

<h4>364</h4>ogsall til sm Due : „ Lad mig fee Tin Skikkelse , lad mia hsre Din Rsft ; thi Din Rsft er sod , og Din Skikkelse er hndiq " ( Hors . 2 , 14. ) . „ Men " , siger Du „ dette gjcelder hans Due , de de fromme , elskelige Sjcele ; i dem kan han have sin Lyst , men ikke i mig eller saadanne som jeg ; thi jeg er en vederstyggelig hynder . " Hvad hore vi ? Stal Gud nu have Behag i Dm egen Fromhed ? Stal Du nu tcekkes Gud for Din egen Skyld ? Fwr Du strax forkastet den trostelige <b>Sandhed</b> , at „ Herren har Behag r fine Gjerninger ? " at „ vi cre benaadede i den Elflede " , ikte i os <b>selv</b> ? „ Men " , siger Dn , „ jeg finder idel Synd og Urenhed hos mig ; dette er da ikke Herrens Vwrk . " Svar : ja just det , at Du finder idel Synd og Urenhed hos Dig , det er Herrens Vcerk . Synden er ikke Herrens Vcerk ; men at Du fsler den hos Dig , det er ikke Naturens Vcerk og heller ikke Djcrvelens . Slangens Indgivelse : I skulle blive ligesom Gud , har opfyldt hele Menneskeslægten med Selvforgudelse . En sonderknust og ( over Synden ) bedrovet Aand er et saadant Herrens Vcerk , at Gud og hans Engle glcede sig derved ( Lul . 15 , ) det er et saadant Herrens Vcerk , som forer fortabte S onner og Dottre i Faderens Favn som driver Synderen til Naadestolen for at indsoobe sig i Kristi Retfærdighed, „ det bedste Klcrdebon " og fee , her er Guds <b>Hjertes</b> storstc Lyst og Glcrde . Den , som er ifort Kristus , den er hellig og herlig for Guds Vie , om han end er den vederstyggeligste Synder i sig <b>selv</b> og i sine egne Vine . Det er om saadanne , Herren stgcr : De hellige , som cre i Landet , og de Herlige , til hvilke al min Lyst er ( Ps . 16 , 3. ) . Kan noget Menneske vcerc hellig eller herlig i sig <b>selv</b> ? Hvad er et Menneske , at det skulde dirre rent , eller at den skulde vwre retfwrdig , som er fsdt af en <b>Kvinde</b> ? Himlene ere ille rene for hans Dine ( Job . 15 , 14. 15. ) . Skulde da Petrus og de andre Disciple vcere rene i sig <b>selv</b> den Aften , da de „ trcrttede om , hvem der var den ypperste " , strax efter Modtagelsen af tvende Sakramenter da de ikke formanede at vange en Time med deres bedrovede Herre i Urtegaarden , da de alle forargedes paa ham , da Petrus forncrgtcde ham m . m . Men samme Aften sagde Kristus tvende scerstilte Gange : Nu ere I rene ganske rene . Men han sagde ogsall Grunden dertil; han talcde om Toning og om sit Ord , som de nu troede ( Joh . 13 , 10. Kap . 15. 3. ) . Og om vi end ikke betcrnke det , saa mener han dog , hvad han har sagt : Hvo , som troer og bliver dsbt etc. eller hvad Paulus siger om Kristus og hans Brud : Han rensede den formedelst Vandbadet ved Ordet . Det er paa denne Grund , han seer Petrus ganske ren og siger om Petrus og hans arme Brodrc : Nu ere I rene . O , stands og tcrnk ! det er jo Kristus <b>selv</b> , Dommeren paa den yderste Dag , som <b>selv</b> siger dette ! Skulle vi da ikke vaagne og sce . at det er Alvor med Kristi Retfcrrdighed , Alvor med , hvad Ordet siger om Renselsen i hans Blod ! Det er forskrækkeligt med vor Vantro og vort Morke ! For os er Kristi Blod ofte et Intet , for vort <b>Oie</b> er Krist .

<h4>383</h4>Da bekymres de og klage og frygte for , at Guds Aano er bleven bedrevet og bortdreven , og at de ere overgivne af Gud . O , om Du vidste , kjcere Sjcel , hvad det er , Du nu erfarer , saa skulde Du inderlig fryde Dig ! Det er intet andet end den fortroligste Leg af den kjcere Frelser . Saaledes pleier han . Hvorledes gjorde han med den kananlriske <b>Kvinde</b> ? Hun kommer tillidsfuld til ham , overbevist af det almindelige Rygte om , at han var en mild og venlig Hjcrlper . som ingen stsdte bort . Hun kommer med en Uor Nod paa sit fjerte og beder om hans Hjoelp . Han gaaer blot bort og svarer hende ikke et Ord . Hun gaaer efter og raaber han gaaer bestandig bort . Omsider rores Disciplene til Medlidenhed med den arme <b>Kvinde</b> , da giver han et Svar , som synes at vcere endnu vcerre end hans Taushed : Jeg er itte udsendt uden til de fortabte Faar af Israels Hus . O , hvilket Stmg igjennem den arme <b>Kvindes</b> <b>Hjerte</b> ! Hun faaer at hore , at al hans Naade itte tilhorer hende : hun er ilke iblandt dem , fom skulle nyde hans Hjcelp . O . kjcere Frelser , o , at Du kan prove Dine kjcereste Born saa haardt ! Men <b>Kvinden</b> lader endnu ikke Modet falde ; hun beder endnu en Gang : Herre , hjcrlp mig ! Da bliver han endnu haardcre kalder hende en „ Hund " og sagde : Det er ikte smukt at tage Bornenes Brod og kaste det for fmaa Hunde . Var det ikke forskrækkeligt ! Her seer jeg , at Frelseren kan paa det dybeste fhule sin Naude og " paa det bittreste bedrove og saare dem , som han inderligst clstcr . Eller hvorledes var det i hans <b>Hjerte</b> , medens han talede disse haarde Ord ? Det see vi snart , naar han aabcnbarer sig . <b>Kvinden</b> tager ham paa Ordet , og naar hun horer , at hnn dog maatte vcere saa meget sor ham som en Hund , saa omfatter hun dette og siger : Jo , Herre , Hunde oede dog ogsaa af de Smuler , som falde af deres Herres Bord . Da kunde han ikke holde sig lcengerc ; nu kommer det frem , hvad han havde i fjertet . Med <b>Kjcerlighcd</b> og inderligt Velbehag udbryder han : O , " <b>Kvinde</b> , Tin Tro er stor . Dig skee som . Du vil ! See nu her , hvad han havde i sit <b>Hjerte</b> , medens han gjorde hende til en Hund ! Idel brccndende KjcrrKghed og Venlighed . Maa ikke dette hedde at skjule sig dybt , at have sin Lyst med Mennestens Born og dog anstille sig sall haard imod dem ? Men , siger Du . her var det vel en Leg . men med mig er det ikke saaledes ; jeg har syndet , jeg har bedrevet Aanden . det er min egen Skyld , at han er vegen fra mig . Hvad hore vi ? Fandtes der vel nogensinde en Sjcrl , som sogte at staa i Naade hos Frelseren og kun at have Naadc paa Grund af hans Fortjeneste , men dog var overgiven af Gud ? Mcerk . en Sjcel . som ikke <b>selv</b> soger at forhverve Guds Velbehag , men soger kun i EvangeliOrd den Naade at kunne tro — fandtes der nogensinde en saadcm Sjcel , som dog var overgiven af Gud ? At Du har syndet og bedrovet Guds ' Aand . det kan vel vcere <b>sandt</b> , men at han derfor skulde vcere vegen fra Dig , naar Du endnu sukker efter Naaden i Kristo , det er aldeles umuligt ! Den . som 2 f Gud er hengivet i et vrangt Sind , den sukker aldrig ester Naade

<h4>418</h4>hvilket fattedes den Mand i Brylluppet , som blev kastet i det yderste Morke . Ved faaledes at blive blottet for al egen Trost og ifores Kristi Retfærdighed faaer Hjertet en saadan Lettelse , Fred og Gloede , Liv og <b>Kjcerlighed</b> , som umulig kan dolges og voere stum , men udbryder i glad Bekjendelse og Frelserens Pris , og her er da den broendende Lampe og den Olie , som fattedes de daarlige Jomfruer . Men om ogsaa Troen endnu er svag og koempende , er mere en rygende Boege end et broendende Lys , bestaaer mere i Hunger end i Moettelse , derpaa kommer det ikke an , naar kun der er en fattig og ene efter Naaden i Kristo loengtcnde Aand kun at jeg itte nsies uden denne Naade , men soger derefter . Summen af alt , hvad Lov og Evangelium skulle virke , er jo , at jeg synes ilde om mig <b>selv</b> og har min Trost i Kristus alene ; det er jo , hvad alt Guds Ord fordrer . Naar dette sindes , saa er der Liv , om end alt andet godt synes at fattes , og alt ondt at flyde over . Men savnes det , saa er der Nod , om der end forovrigt var Overmodighed paa alt godt . Dommer ikle efter Anseelse , men dommer en ret Dom . Nt Synd og Skrsbelighed , Mangler baade i Forstand og Levnet hoengc ved os , derfor bor vi ikke fordomme hverken os <b>selv</b> eller andre ; Kristus boer ogsaa i stor Skrobelighcd , saa at „ det er den stsrste Kunst under Solen at see Kriftns i de svage " ( Luther ) . Derfor , saaloenge der blot sindes dette ene voesentlige : en fattig og efter Naade i Kristo hungrende Aand , saa er der altid Liv . Derimod , om jeg end groemmer mig tildode over en syndig Gjerning , en begaaet Feil , men ellers er tilfreds med mig <b>selv</b> , ikke lider af Hjertets Ondskab , Koldsindighed, Haardhed , Sikkerhed , Hykleri osv. men i Almindelighed er selvtilfreds og sikker , saa er det intet godt Tegn ! Om nu dette er en „ haard Tale " , saa er det dog Kristi Loerdom . Vi maa ikke gjore os Veien hverken bredere eller smallere, end den i <b>Sandhed</b> er . Desuden er det kun haardt for den stolte Adam : men for fattige Syndere er det ikke haardt , at Jesus kun frelser det , som er fortabt . Voer da ikke roed for Lyset , om det end belyser en ulykkelig Tilstand hos Dig ! Det skeer til Din egen evige Gloede og Salighed . Om Dit <b>Oie</b> og Dit <b>Hjerte</b> bliver aabent for <b>Sandheden</b> , saa bliver Dit Liv herefter langt lykkeligere, end det har voeret ; Du gjor store og salige Opdagelser . Det begyndes vel med Sorg og Bekymring , men det er en „Bedrsvelfe efter Gud , som virker Omvendelse til Salighed , som ikke fortrydes " . Saaledes gik det ogsaa med vor kjoereEnkc i Sarepta . Der blev stor Sorg , da hun saa sit elskede Barn ligge dodt paa Baarcn , og isoer derved , at hun deri saa en Guds Hjemsogelse og sortes for Guds Dom med fin „ Misgjerning " . Der var saaledes en dobbel Sorg . men den forvandledes ogsaa til en dobbel Gloede , nemlig ikke blot den , at hun paany modtog sit elskede Barn af Guds Haand , da den dode Dreng atter blev levende ved Elias ' Kraftbon sv . 19 — 23. ) , men hvad ' der i Soerdeleshcd blev

<h4>746</h4>Uke forarges . " hvilket skete nogle Timer , forend han fornægtede sin Herre ; og David , da han gik paa Taget og kastede letsindige Blikke paa <b>Kvinden</b> uden at frygte nogen Fare eller naar man ilte mere har nogen Omsorg for at tiltage mere og mere i det gode ; naar Naaden i Kristo ikke glceder Hjertet , naar Ordet og Bonnen ikke mere smage osv. Men hvad der herved udmcerker den redelige Aand og stiller en levende Kristen fra en dod , det er , at den fyrste snart kommer i Bekymring over sig felv , snart faner et vcrlkende <b>Blik</b> af sin Herre , saa han gaaer ud og groeder bitterlig , eller om det er kommet saa langt med Sikkerheden , at Gud mua anvende ydre Midler til Tugt og Revselse eller sende en straffende Nathan , saa lader han det blive sig til Gavn , tager Advarslen og Bestraffelsen til <b>Hjerte</b> , bekjender sin Synd og Ssvmahed og vil blive bedre osv. Derimod er det et Tegn paa , at Ssvnen oq Sikkerheden er gaaet over til Dod og Forftottelse , eller Tegn paa en falft Kristen , naar man fremdeles er tilfreds og tryg , ikke lader sig advare , men enten som Judas Iskarioth vedbliver i nogen bevidst fornægter , stjuler og forsvarer den , eller som de daarlige jomfruer udvortes i alt ligner de vise , men dog i Hjertets skjulte Dyb savner Naadens Liv og Erfaringer og i Taushed lader det bero indtil videre , til — Dsren er lukket . O , hvilken forskrcekkeliq Tilstand , naar et Menneske ikke mere har Evne til alvorlig Besindelse, ikke har Evne til at standse og betcenke de alvorligste Ting , ikke kan frygte eller mistoenke sig <b>selv</b> ! Men saaledes er den mennestelige Natur en forskrækkelig <b>sand</b> Betydning af den Dsd . som stulde blive Syndefaldets Folger „ Du skal os Dsden" ( 1 Mos . 2. 17. ) en <b>sand</b> Betydning af Beskrivelsen i Rom . 3 , 18 : „ yerrens Frygt er itte for deres Dine . " De hore , de lceie . de tro , at tusinde andre ere blevne bedraqne , imod deres egen Formodning bedragne til deres Ulykke , men wnne ikke frygte for , at de muligvis <b>selv</b> ogsaa kunde blive det . De lcese , de hore udmattende Tegn paa deres egen Tilstand , men slaa det hen i Veiret , og vende sig til Smaating . Sce her nogle Exemplcr : Der var en from <b>Hustru</b> som en Morgen sagde til sin Mand : „ At , min Ven . ieg horer Dig aldrig mere nu som i de forste Tider af Dit Naadeliv at tale med Glcede om Kristus og Troens Liv , eiheller mccrker zeg , at Du lider af Din Skrobelighcd , eller at Du behover at lcese noget Guds Ord . Sig mig oprigtigt , hvorledes Du nu har det med Gud og Din Sjcrl ; har Du nogen Omgang med Den skikkelige Mand rodmede lidt herved , mcn ' bad hende ye , pcent ikke at forurolige fig over hans Sjælstilstand , vendte lig tn andre Ting og nynnede paa en forfcenqeliq Vise . See . detti ) er sikkerhed ; han begyndte ikke <b>selv</b> at bekymres over sin tilstand og undersoge . hvorledes det virkelig kunde staa til dermed, iyvorvcl der lunde voere stor Anledning dertil ; men han slog dei yen r Benet . Dette er Tegnet paa . at Herrens Frygts Aand 19 ' '

<h4>959</h4>lade ikke Hjertets haarde Klippe sønderknuses kortelig , ikke cre oprigtige og uden Falskhed , men endnu have saa mange urene Hensigter , ikke lade Viet vcere enfoldigt , men endnu soge et og andet i Verden og ville smugle saa meget ind med sig ' samt ere altfor blodagtige imod deres Kjod og eftergivne for dets Lyster , saa kunne de ikke komme til nogen <b>sand</b> Fred og Vished . Saadanne Sjoele maa ophore med at undskylde sig og ikke taste Skylden paa noget andet udenfor sig <b>selv</b> , men tro , at de <b>selv</b> ere Skyld med deres fordærvelige Egenvillie ; de bor tro , at det Sted og de Omstændigheder , i hvilke de ifolge Guds Kald og Styrelse befinde sig , og hvori de dog ere blevne kaldte og opvakte , visselig ere de bedste for dem , ja at ingen kan gjore dem Skade eller vcere dem til Hindring , naar de blot ville tragte efter det gode , at alt maa tjene dem til det bedste oq befordre deres Salighed , at Kristus vil hjcelpe alle , fore dem alle til den sande Fred i ham . Men de maa komme dertil , at det virkelig er Alvor for dem med Omsorgen for deres Sjcel , at ingen Ting ligger dem saaledes paa <b>Hjerte</b> , lom at de maa redde deres Sjoel og blive visse paa deres Salighed. De maa , saa at sige , ikte vcere som Vcirhancr , ikke vcere saa ustadige , ikke oftfore sig vel iblandt de fromme , medens de flikke sig Verden lig , naar de ere iblandt dens Born . De maa vcere rcrdde for al falsk Tro som for deres stsrste Fjende , ikke tale oq foregive mere eller antage et bedre Skin for Mennesker , end de : Virkeligheden ere , og altsaa ikke ville blot behage Mennesker oq have disses Vidnesbyrd og Bifald ; thi det gavner dem intet , men de maa vandre for Gud og tragte efter hans Vidnesbyrd i Samvittigheden. Naar de komme dertil , det vil sige , lade Herren opvcekke sig dertil , saa skal det gaa dem vel . Og saadanne Mennesker, som af Naturen have et let Sind , skulde i Almindelighed snart komme dertil , om de blot vilde begynde og fortscctte alvorligt og oprigtigt og ikke give sig Ro . forend de havde fundet Fred i Haar alene ; men da de ilke gjore dette , saa nodes Gud til at cenglte dem med sin Tugt baade indvortes og udvortes , indtil de blive oprigtige og uden Falskhed , det er , ikke ' mere gjore nogle og Betingelser , men ville give Jesus hele deres <b>Hjerte</b> og ikke ville beholde noget skalkagtigt <b>Oie</b> eller nogen uren Hensigt , cn Den anden Klasse af Mennesker , som gaa lcenge i Kval og Morke og savne Troens Vished og Fred , ere saadanne , som vel Zunne vcere ganske alvorlige i deres Sogen efter Guds3nge , men ere meget haardt fcengstede af nogen vis Lssnke , nogen afgndist Kærlighed eller nogen gammel Skjodcsynd . som holder dem fangne . <b>Kjcerligheden</b> til denne Afgud eller Skjodesynd er saa Mrk og mcegtig over dem , at de ikke kunne forståa , om de virkelig vllle vcere den kvit , og derfor udelukke de sig bestandig fra Naadcn , da denne jo ikke kan tilhore saadanne . som Me ville stilles ved Synden , og naar da Hjertet ikke nogensinde i yimmeiit ' Hkat , ikke har faaet nogen overveiende Lykke og Gla-ds

<h4>1446</h4>imod Nisten . Om de forste siger Skriften : Jeg formaner Eder , jeg den bundne i Herren , at I vandre vcrrdMN det Kald , hvormed I ere kaldte alle Lemmer have ikke den samme Gjerning som Herren har kaldet enhver , saaledes vandre han ; Omskærelsen er intet og Forhuden er intet , men at b > vare Guds Bud ; som enhver . Brsdre , er kaldet , saaledes blive han for Gud m . m . ( Ef . 4 , 1. Rom . 12. 4 ; 1 Kor . 7 , 17 — 24 , m . fl. ) . Om de fcrlles eller almindelige Pligter hedder det : Alt hvad I ville , at Menneskene skulle gjgre mod Eder , det qjsrer ogsaa I mod dem Du skal elske Din Nwfte som Dw <b>selv</b> <b>Kjcrrlighed</b> er Lovens Fylde ( Math . 7 , 12. Rom . 13 , 9.10 . ) . Dog bor det bemærkes , at de fleste Kjcrrlighedstjenester imod Ncesten udsves just i Kaldets Gjerninger , og dog er der kun faa Mennesker , som ret betoenke , hvorledes Gud just i vort Kald har rettet sin forste og fornemste Fordring til os . Man bor derfor navne sine <b>Oine</b> og see . hvorledes der er en Guds Orden og Indretning lige fra Guds Trone i Himlen og til det nederste Sted paa Jorden . Gud , som er „ Ordenens Gud " , har anordnet det saaledes . at den ene stal voere Regent , den anden Undcrsllllt , den ene Husherre , den anden Tjener , den ene Fader eller <b>Moder</b> , den anden Barn . den ene rig , den anden fattig , den ene gift . den anden ugift , ligesom det ene Lem i et Legeme stal vcrre Ore . det andet Ore . det tredie Haand , det fjerde Fod osv Nu er det Guds fmste Villie og Fordring , at enhver passer sin Gjerning , at den regerende regerer vel , at den lydende adlyder tro , at <b>Oiet</b> seer og overlader til Oret at hore , at Foden ganer og overlader til Haanden at arbeide . O , hvilket saligt Levnet stulde ikke indtroede paa Jorden , om enhver betcenktc , at det er Guds forste Villie og Fordring angaaende vort Levnet , at enhver gjor fit Kalds Gjerning , at det er en <b>sand</b> og hellig Gudstjeneste , som behager Herren vel , at man midt i Husholdningen og i de NmpWle Gjermnger staaer i et helligt Tempel og udforer den behageligste Gudstjeneste . Men ak ! hertil fordres et aandeligt <b>Oie</b> lom ikke blot feer paa det ydre , mm seer Guds Anordning og Befaling . Om vi med vore legemlige <b>Oine</b> saa Gud for os og hsrte ham at begjcrre en Tjeneste af os , om den end i sig <b>selv</b> var den allerringeste , faa var den os da ikke mere ringe , men fik strax et hmt Vcerd , og vi stulde udfore den med storste Lyst . blot fordi vi kunde tjene Gud dermed , som Luther siger : „ Om Gud med sine Engle en Dag befogte Dig og blot befalede Dig at feie Gulvet , saa skulde Du vel blive saa glad derved , at Du ikte vidste , hvorledes Du stulde stille Dig an derved sandelig ikke for Gjermngens Skyld , som jo i sig <b>selv</b> var ringe , men for hans Skyld som befalede Dig den . " Men nu har jo Gud virkelig givet Dig saadanne Befalinger , naar han f . Ex . har ladet Dig blive en Tjenestepige eller et Barn i Huset ; thi til dem har han sagt : Vwrer Eders Herrer underdanige og lyder dem — ha-dre DW Fader og Din <b>Moder</b> , altsaa , gjor , hvad de befale Dig at

<h4>1473</h4>sig strengt til Bogstaven og tage derfor vare paa den Herre Jesus for at see , om han vilde bryde Budet . <b>Selv</b> havde de ikke hjulpet den syge , om det end havde kunnet stee med en Skefuld Vin Men Kristus trænger ind i Lovens Mening efter Kærligheden og hjcrlper dristig den vattersottige Mand samt viser dem en soleklar Aarsag , hvorfor han gjsr saaledcs , og vil sige som saa : „ Det er vel befalet at helligholde Sabbaten , men naar det kommer an paa <b>Kjcrrligheden</b> , da maa Lovens Strenghed staa tilbage " , som han ogsaa siger paa et andet Sted : Sabbaten er skabt ' for Menneskets Skyld og ikke Mennesket for Sabbaten . „ Ligesom nu Kristus gjor her med Sabbaten , saaledcs bor ogsaa vi gjore med alle Love og ikke holde dem videre , end de tjene Kærligheden . See her endnu et Exempel fra vor forrige Tid under Pavedsmmct : En Mand har gjort etLofte om en lang Pilegrimsreise . Nu lyder Budet saaledcs : Den , som gjsr ct Lsfte , bor ogsaa holde det . Men denne Mand har <b>Hustru</b> og Bsrn , Tjenestefolk og Husholdning hjemme . Hvorledes bor han nu forholde sig , stal han holde sit Lsfte , eller stal han blive hjemme og forsorge <b>Hustru</b> og Born ? Dommer <b>selv</b> , hvilket der er mest nodvendigt og overensstemmende med <b>Kjcrrligheden</b> . Jeg holder for . at det er nyttigere , han bliver hjemme , ' arbeider og forsorger sit Hus . " Saa langt Luther . See nu her : Budene om Sabbaten og om at holde Lofter ere jo fortracffelige Bud til stor Nytte for os Mennesker , - thi toml , om vi ingen Sabbat havde , eller om <b>Sandhed</b> og Ordholdenhed aldrig agtedes ? Hvilket stort og almindeligt Onde var lkke dette ! Og dog vise de anforte Exempler , hvorledes <b>Kjcrrlighed</b> og Nedfald stundom maa ophceve de gode Bud , om f . Ex . et ondt og daarligt Lsfte skulde opfyldes , faa at deraf vilde komme et dobbelt Onde . Der stal da <b>Kjcrrligheden</b> va-rc Rettesnor . Men det gaaer ofte , fom Luther her anmccrkcr : „ Naar Profeterne kom og vilde udtyde Loven efter <b>Kjcrrligheden</b> og sagde : Saa og saa mener Herren , saa og saa bor det Bud forftaaes . da vore de falske Profeter strax tilrede ; nu holdt de sig meget strengt til Texten og sagde : Der og der staaer det strevet , det og det er Guds Ord ; man maa ikke fortolke det anderledes osv. Hvor meget nu end de rette Profeter raabte . at saa og saa burde det forftaaes , saa vilde det dog ikke hjcrlpc . " Ja saaledcs ganer det endnu rdag og er visselig et meget betcenkeligt Tegn paa en falst Aand , som ikke oprigtig sporger blot efter herrens Villie . men hellere soger at besmykke sit eget Sind og Rcmd " Gud hjcrlpe os at vcere oprigtige ! Vi have hermed berort selve Hemmeligheden og Hjertcpunktet , hvorpaa det tilsidst stal bero . om man stal forståa ' og erkjende Herrens gode , behagelige og fuldkomne Villie , nemlig et oprigtigt <b>Hjerte</b> , som lun vil vide , hvad Gud vil . Her er Sagen : her er Knuden : hervaa beroer det ! Mange kunne have en saa stor

<h4>1483</h4>dets Gudstjeneste . At deres Proester forvendte Loven ( Math , 5. ) og korsfæstede Kristus var ham ilke tjcert , kunde heller ikke bortjage ham : thi hvad gjorde ikke Paulus for Frelse ? „ Paa det jeg kan vinde dem " , at „ jeg dog for altmg kunde frelse nogle . " See . dette var hele hans Sag . Men naar han derimod var ude iblandt Hedningerne , da taug han aldeles stille om den levitiste Lov og brcendte af hellig Iver mod de falske Apostle , som bekymrede hans Galater med Lcercn om Omskærelsens Nødvendighed. Alt gik kun ud paa at beholde alle ved Enfoldigheden i Krifto og „ ikke gjsre nogen Forhindring " for det eneste saliggørende Ord om ' Omvendelse og Tro . See her , hvorfor han i Antiokien bestraffer Petrus for det samme , som han <b>selv</b> gjor i Jerusalem ! Da Petrus i Antiokien var omgiven af en Skare omvendte Hedninge og dog rettede sig efter nogle faa jsdiste Kristne , som kom fra ' Jakob i Jerusalem , hvor de troende endnu havde Nidkjcerhed for Loven , saa „ irettesatte Paulus ham for alles <b>Oine</b> . " See Gal . 2 , 11 — 12. Men naar Paulus tom til Jerusalem, hvor han blot havde loder omkring sig . da gik han <b>selv</b> i Templet og rensede sig m . m . SeeApg . 21 , 17 — 27. I Brevene til de Romere og Galater bestraffer Paulus , at de hcengte ved Omskærelsen : men naar han tommer til Lystra og finder den troende Discipel Thimoteus . „ hvis Fader var en Groeter " , hvad gjor han da ? Johan den samme Paulus omstår ham <b>selv</b> , Og hvorfor ? „ For Isdernes Skyld , som vare paa disse Steder ; thi alle tjendte hans Fader , at han var en Grader " lAvg . 16 , j . Saaledcs lader han sin klare Forstand om det rette her vige " for Kærlighedens Nidljcrched „ for at gjvre mange salige . " Hvor klart han end vidste , hvad der i sig <b>selv</b> var rettest , gjorde han dog kun det , som var det nyttigste , nemlig for ikke at bortstodc disse loder fra sin Prcediken . Saaledes gjor den sande <b>Kjcerlighed</b> og Visdom . Og om end nogen vilde heri see Dobbelthed , Mennestebehag og Hykleri og sige : Paulus vandrer ilte efter nogen Vis Regel , saa stal den hellige Apostel med Aandens Vidnesbyrd svare : „ Kristi <b>Kjcerlighed</b> tvinger mig saa ledes " „ paa det jeg maa vinde desto flere . " Og den , som i <b>sand</b> Tro vil eftcrfslge de nu anforte Exempler af Moses , Kristus og Paulus , maa troste sig paa samme Maade , om han faaer samme Beskyldning . Ogsaa iblandt os opstaa mange Meninger . Den ene bekymrer sig over Sabbaten , fordi den ikke holdes paa den forestrevne syvende Tag , paa hvilken „ Gud hvilede fra alle sine Gjerninger " , men paa den fsrfte . paa hvilken Kristus opstod — og han sporger : Hvor byder Guds Ord denne Forandring ? Den anden bekymrer sig over de smaa Borns Daab og sporger : Hvor er den befalet ? Den tredie bekymrer sig over , at <b>Kvinden</b> beder med übedcekket Hoved , eller at noaen spiser Blod osv. efterdi der ogsaa i det nye Testamente er talet derom ( 1 Kor . 11 , Apg . 15 , 20. ) . En fjerde

<h4>1720</h4>som Gud har stjcrnket os i Kristus , men kan see Dine Medmennesker at vandre imod en evig Dod og Fordommelse i aandelig Dod og Sikkerhed , og Du gjor ikke det ringeste til deres Vcrkkelse ; Du taler med dem om alt andet og veed i Stilhed deres tilstundende Ulykke . O , vaagn op , bed Gud om en saa oprigtig Kærlighed , at den bestandig arbeider i Dit indre , saa at Du fßger nogen Leilighed og Maade sor at tjene Dine Medmennesker til Frelse . — - , Du , som troer og er benaadet , men seer Din fattige Broder at lide Nod , og Du har Midler til at hjcelpe ham , men lukker Dit <b>Hjerte</b> til ; hvorledes bliver Guds <b>Kjcerlighed</b> i Dig ? Droeb her Dit Kjod , som vil beholde alting for Dig <b>selv</b> og Dine . See , as saadcmne Exempler kan Du mcerke . hvad det er , at der i Troen sindes Dyd , hellig Krast og Iver . Nu siger Apostelen : „ anvende ? al Eders Flid " , bed Gud om hans Helligaand til at tro og elske saaledes , at det beviser sig i hele Eders Levnet . Gud forlade os alt , hvad der feiler heri , og opfylde os saaledes med Glceden og Kraften as de storste og dyrebare Forjættelser , han har givet os , at nogen virkelig Forbedring maa folge deraf ! Men Apostelen siger videre : Dg , i Dyden Kundswb . Et besynderligt Ord , men rigt paa velsignet Lcerdom og dette ogsaa for de mest kraftfulde og nidkjoere Kristne . Ordet „ Kundskab " betyder i Grundtexten ogsaa Visdom , Forstandighed . See , her er Lcerdommen . En <b>sand</b> Kristen stal ikke blot udove Dyd , hellig Iver og Kraft , men stal ogsaa see til , at han udover den paa den forstandigste Maade med Visdom og Indsigt ; han maa ikke gaa blindt tilveie , men med Bon om Oplysning af Herrens Aand bestandig tcenke efter , hvad der er det viseste og tjenligste ved ethvert Tilfcelde til at fremme Guds Mre og Menneskers Velfcerd . Er ikke ogsaa dette en hoist vigtig Formaning ? Ja naar vi ret betcenke , hvor tusindkunstig Satan er til at forvilde os og hvor stor Skade og Forargelse , det afstedkommer , naar en Kristen i velment , men uviselig Nidkjcerhed begaaer nogM „ Daarlighed i Israel " , « giver Bagtaleren Rum " og svage SMe Anstod — o , hvor vigtigt maa det da ikke vcrre os uophorligt at bede Gud om hans Lys og med Rcedsel for vor egen Aand uophorligt sgge at erholde mere og mere Kundskab om , hvad „ Guds gode . behagelige og fuldkomne Villie er . " De . Kristne behove sandelig at vcere „ fulde as <b>Oine</b> baade fortil og bagtil " , som der staaer om de sire Dyr for Lammets Trone ( Aav . 4 , ) . For nu at see af nogle Exempler i Bibelen , hvad dette vil sige , saa lad os forst betragte Apostelen Paulus , som giver os det mest fremstaaende Exempel i denne Henseende . Denne Apostel var , fuld af Tro og hellig Nidkærhed , saa at han endog havde . . arbeidet mere end alle de andre " , men til samme Tid var han saa rced for en uviselig Udovelse af sin hellige Nidkjcerhed , han var saa opmcerksom paa sine Omgivelser , paa Forhold og Omstændigheder, vaa hvad der var det mest velgjorende for andre , at man kan sige i <b>Sandbed</b> , at han gjorde sig til en frivlllig

<h4>1792</h4>Eller ere vi omsider forte omkring af nogle „ klogtige Fabler ? " „ Guds Arvinger og Kristi Medarvinger ! " Visselig see vi , at Gud , tabte Mennesket til Arving og Herre over alt det , han havde skabt ; visselig forståa vi , at „ <b>Kvindens</b> Soed " skulde i alle Henvender gjenoprette , hvad der mistedes ved Faldet ; visselig hore vi ham <b>selv</b> sige til sin Fader om sine strobelige Disciple : Zeg har givet dem den Herlighed , som Du har gibet mig ; vlssellg hore vi ham at tale om sine Brodre og sige : Min Fader og Eders Fader , min Gud og Eders Gud ; men vi kunne V faa det ind i vor Forstand . „ Guds Arvinger og NNNI Medarvinger ? " Det maa da vcere overmaade rige Arvinger de , som ere Kristi Medarvinger ! Dog er det ogsaa overmaade stort at vcere Guds Born , og ere vi Guds Born , saa ere vi jo Kryn Brodre og Sostre , og ere vi Kristi Brodre , da ere vi ogsaa hans Medarvinger . Nu er vel alt dette meget for stort til , at vi kunne ngtig tro og fatte det ; men hvad gjcelder ? Det er dog Guds eget evige Raad og hans egne Ord . Og det er ikke mindre Itort , at Guds Son blev et Menneske og vor Broder . Mm dette er nu dog alligevel steet . Og derfor , stjsndt vi ikke kunne fuldkomment tro dette , er det dog lige <b>sandt</b> og vist . da han , som har mort opsagt det , hedder : „ Trofast og Sandfoerdiq . " Han <b>selv</b> wgde : har givet dem den Herlighed , som Du har giver ung og sagde dette om dem , som endnu gik paa Jorden llUillll havde han allerede givet , altsaa eiede de allerede denne Herlighed , stpndt den endnu var forvaret til dem , og de Ve M den . Og hvad denne Arv , denne Herlighed indeholder , det er noget som „ intet <b>Oie</b> har seet . intet Ore har hort , og som i ! e er opkommet i noget Menneskes <b>Hjerte</b> . " som ogsaa Johannes vi Guds Born , og det er endnu ikke aabenbaret . hvad vi stulle vorde ; men vi vide , at naar han aabenbares , da stulle v : vorde ham lige . " Men nu naar jeg begynder at tro saadanne Ting og Wer en med Gud fortrolig Barnestabsaand , da kan jeg umuligt blive paa Afstand fra den hulde Fader og den dyrebare da jeg at tale et fortroligt Barnesprog med Ida ere Souner , har Gnd N " / ? Z Sons Aano : Eders Hjerter , som raaber : Abba , ( Gal 4. 6. ) Og her er nu det tredie , som horer til vor . samfund med Gud , nemlig Omgangen , den fortrolwe Omgang med yam . „ Denne fortrolige Omgang med Gud er en barnlig og tillidsfuld faavelsom hellig , <b>kjcerlig</b> og lydig Omgang , oa man omgaaes wm et Barn med sin Fader og strax med Anger beder om saa fnatt man har forfeet sig , men dog beyotoer den barnlige fortrolighed , og derfor vedbliver at omqaaes medlam og vil kun vcere hans egen . " Af det troende Barns <b>Hjerte</b> gaa utallige og uudsigelige Sukke til Frelseren , hvad det end , oretager Ug ; thi hans Naade og Ncervcerelse er Hjertets man vandrer for ham , vogter paa hans <b>Oine</b> og soger

<h4>1827</h4>kan aabenbare sit <b>Hjerte</b> for nogen og dette steer ogsaa med en alvorlig Sjcel men om end ikke nogle usædvanlige eller grove Synder ere forhaanden , er det jo dog altfor svcert at kunne tro Guds Venskab , naar man saa bestandig maa sinde Synd og Urenhed i alt , hvad man tcemer , taler og gjor , og endelig ( hvad der ofte er vcerre end alt andet ) dette uhjcelpelig dode og falste <b>Hjerte</b> , som hverken kan helt og bestemt hade Synden eiheller ret angre den eller tro og elske Gud ; men tvertimod er det <b>Kjcerlighed</b> til Synden , Letsindighed og Hykleri alt , hvad der fornemmes . Naar nu hertil kommer , at hele vort indre er aldeles gjennemtrcengt af Lovsind , saa at samvittighed , Fornuft og Folelse alt taler kun om Ret og Fortjeneste , medens derimod en virkelig Naade er aldeles fremmed sor vort Sind o , hvilken trykkende Tyngde , hvilken Brydning , hvilken Kamp paa Liv og Dod , der da opstaaer i Sjcelen ! Gud staaer blot med Retfærdighedens Dommerblik hellig og forskrækkelig for dens Vie , og alle Noadetilsigelser kastes tilside blot ved dette ene : Du veed med Dig <b>selv</b> , hvad Du har gjort , hvorledes Du har voeret og er . tvertimod Guds Ord . Hvorved stal jeg da berolige mig ? Alting er uholdbart og flyver i Veiret som lette Avner , om man ikke da kan faa fast Grund i Guds eget urokkelige Salighcdsraad , Guds egen almindelige, evigt gjceldende Afgjorelse af Sagen . Og hvad er da dette Guds Evighedsraad og faste Beslutning om vor Frelse ? Det er . hvad vi just have at betragte i dette Sprog : Dersom tn vandre i Lyset dersom vi blot komme , lade os bestraffe , men oasaa fly til Naadestolen med vor Elendighed have vi Samfund med hverandre , og Jesu Kristi hans Ssns Blod renser os fra al Synd . ~ . > Tcenk nu Du , som er en ret stor Synder , hvad veier Du imod dette , som her staaer ? Giv Agt paa , hvorledes Apostelen taler ' Og sor at Du maatte i noaen Maade mcerke . hvad dette store Forsoningsmiddel gjcelder , ville vi , betragte hvert Ord i dette dvrebare Sprog . Betcenk forst , hvem Personen er . som her tales om Jesu Kristi , Guds Ssns , Blod . Tcenk , Jesus Kristus den saa lcenge forud lovede „ <b>Kvindens</b> Soed " , Herrens Salvede ( Kristus ) „ hvis Udgange er fra fordum , fra Evigheds Dage " , men som dog blev fodt i Bethlehem ( Mik . 5. ) da Engle sang : Skyen : Eder er idag en Frelser fodt . den Herre Kristus oa videre : Han stal hedde Jesus ; thi han stal frelse sit Foll fra deres Synder . Denne Herres Jesu Kristi Blod icenk , hvad det gjcelder ! Hvilken Kraft skulde ikke dette Sprog have , om der kun stod saaledes ! Men der staaer endnu noget herligere deri . Apostelen vil med Flid paapcge Personens hoie Vcerd og Betvdnina , derfor har han endnu indlagt tvende vigtige Ord : Guds Sons - . Jesu Kristi , Guds Sons Blod . " Saa fo g da Apostelens Fingervisning og see paa de Ord : Guds Ssns ! er o t muligt , er det <b>Sandhed</b> ? - Guds Son ! - Og Guds Son udgyder sit Blod for os ! - Er det <b>Sandhed</b> ? Ja er dette

<h4>1852</h4>og det . som derimod er Aandens Frugt , eller som Kristus her antyder , en Frugt af Foreningen med ham . - For nu at fremstille et talende Exempel ftaa , hvad der er Aandens Frugt , saa lad os blot betragte dm forste af de Frugter , som Avostelen opregner paa det anforte nemlig KjoerUgneden. Nu har derhos Kristus <b>selv</b> i samme Tale hvor vore Tcxtord staa , udtrykkelig sagt : Derpaa stulle alle kjende , at I ere mine Disciple , dersom I have indbyrdes KMrltghed . Og vi vide , hvorledes Johannes i sit forste Brev uafladelig taler om Kjeerlighed til Brsdrene som det udmærkende Tegn paa Guds Vorn Vi maa da forst tale om den broderlige Kærlighed . 6 r det ikke <b>sandt</b> , at mangen troende nu allerede tcenkcr , hvad M manae svare , naar man taler om denne <b>Kjcerlighed</b> : „ Hvad er det for et Tegn ? Kjccrligheden til de troende den er en 3 Sag ! Om nogen forbod mig at elste dem , det vrlde bkve tunat men at jeg elsker dem , som soge Gud , det er ganske nnt Hlettes T ? ang / d ' t gaaer af sig sel ° " ° sv- Men mme Brodre ! Verden synes ikke saaledes ; Hyklerne synes M saaledes , Mose Disciple synes ikke saaledes . For alle disse er det lkke nogen w Sag at elste de troende . Elste nogen Vis Krrsten paa Grund af noget legemligt gode , hvorved han har de- elste deres eget Parti det kunne de ; men tnertellg elste alle . SM eog s / ge Gud , det kunne de ikke . knnne snarere efterabe alt andet i Kristendommen end denne chcerkghed Her blive de aabenbare ved et dybt , ofte skjult , men uformcertt fremstillende Fimdstab imod de troende , i Scerdeleshcd imod dem , om ere mest nidkjcere for Kristus og hans Mre , et Fiendstab hvorved de med speidmde <b>Blik</b> opsoge , forstyrre og udbasune alt dct fe laatige , som kan opspores i de Kristnes Hob , og derover over et kjcert Fund . Saaledes gjsr ikke K > r - Nabedm dm Mder sta Me over Uretfærdighed , men gwder ftkved SMdhcd - den wrer Me Nid , opblceses ikte " ( i Kor 13 , ) Om nogen Kristen er falden , saa Me de andre Knstn derved , soge at stjule Synden , nedtysse Talen derom og blot : Kwliqhed ophjoepe den faldne . Saaledes Sier Kjoer lghedm , Ke Apostelens Ord : Naar et Lem lider , saa Ude alle Lemmerne med. Da nu denne <b>Kjcerlighed</b> aldrig Kgger : Natmm - hi i al Menneskenatur ligger tvertrmod w nys omtalte Fjendskab , Slangens Fjendskab imod <b>Kvindens</b> saa svornes Naar fik Du denne <b>Kjcerlighed</b> , om nu er Dig saa let , at dm er Dw anden Natur , idin Hjertetrang ? Naar ftt Du dm Var dct ikke , da Du elv om elcndlg og fordomt Ne og fandt Din Frelse i Kristo ? See , det var altsaa m Frugt af Din Forening med Frelseren ; det var Aandens Frugt Og derfor er denne <b>Kjcerlighed</b> Dig faa let at Du synes thi Du ' behover ikke at arbcche m ^ for at tvinae det dertil ; dct er ikke en Lovens Ojermng . , Rn Frngt . Seeda , hvad det er Me atomer

<h4>1877</h4>Ord og hans Nand ; Du er maastee „ ustadig i Din Gudsfrygt " , og det er aldeles imod Ordet . Du elsker ikke Gud rettelig , men er koldsindig imod ham og derimod maastee meget varm for det eller det Menneske ! Du frygter ikke Gud ret alvorligt , men er maastee helt letsindig ; Din Aarvaagenhed og Strid imod Synden er ikke stor , og Din Anger heller ikke . See , der er Aarsagen til , at Du har en ond Samvittighed . Og hvorledes stal det da gaa med Dig , arme Synder ? „ Jo , jeg kan blive fordsmt ; der er mange fsr mig i Helvede . " Arme Sjcrl , stjcelv for , hvad Du siger ; thi dette er dog noget forskrækkeligt ! „ Ja visselig ; men hvad stal jeg gjore ? Fare til Helvede det kan jeg , men forandre mit <b>Hjerte</b> kan jeg ikke ; ieg er , som jeg er . " Ja da er Du jo aldeles fortabt . „ Ja det er jeg . " Men har Du da ikke hort om en uendelig Kjcrrlighcd hos ham , som siger : Jeg sletter mit Liv til for Faarene ? „ Jo , og det er jnst denne <b>Kjcrrlighed</b> , der stal brcende paa min fordsmte Sjccl i al Evighed ; thi just denne <b>Kjcrrlighed</b> gjor min Synd mere uvccrdig , og Gud er jo dog lige saa retfærdig som kjcrrlighedsrig . " O , see da her , hvor fortabt det er med den Sjcrl , som i Kristi Dod kun seer et Kjcrrlighedsbevis ! See , hvor fortabt den er , som med al Kundskab og alle Betragtninger over Kristi Lidelse endnu har Dcekket hvilende over det Allerhelligste deri ! Saa loer da engang , at Du endnu ikke kjender Betydningen af Kristi Dod , og lcrr at see noget bedre ind i Kristi egentlige Mrinde og Mening med sin Dod . Loer dette , at Du ikke mere folger det forvendte <b>Hjerte</b> , som siger : „ Kristus og hans Dod forstaaer jeg nok , dermed er alt vel ; derpaa er ingen Feil ; men det er i mig <b>selv</b> , det mcm gjores godt , det er mig <b>selv</b> , jeg mcm see paa . " Ved at betragte Kristus paa den Maade maa Du evindelig blive i Din Dod og Fordommelse . Lad os da sige Dig , at saa <b>sandt</b> som det er , at Feilen er hos Dig og ikke i Kristi Forsoning , saa falsk er det , at Du derfor maa see paa Dig <b>selv</b> og arbeide paa Dig <b>selv</b> , indtil det onde hos Dig bliver afhjulpet . Lad os sige Dig , at just fordi Du har al Din Ovmccrksomhed henvendt paa D : g <b>selv</b> , derfor er det saa ilde med Dig . Lad os sige Dig , at alt , hvad der fattes og gjsr Dig usalig , kan ikke afhjcelpes paa anden Maade , end at Du begynder anderledes at „ see pall Jesus " begynder at ane , at Du ikke kjender ham , og derfor blot begynder at soge at faa noget saadant <b>Blik</b> ind i hans Dods Betydning , som Du endnu ikke har havt . Du stal lcrre , at det er saa aldeles fortabt hos os , at Gud nodvendig maattc gjvre det godt helt udenfor os , at Gud maatte gjore i en anden Person alt , hvad der burde gjsres af Dig , og at det var dette , som skete i Kristus , i hans Lovovsyldclse og hans Dod , samt at Du nu lige fan fordcervet og fordsmt , som Du er , maa faa alt som Gave i Kristus , mcerk uden at Feilen endnu er afhjulpen i Dig . Nu ville vi gjentage de Ord : leg fcrtter mit Livstil for Faarene , og betragte dem noget noiere . Du veed af hele Skriften ,

<h4>1949</h4>ogsaa for os vcere det eneste store og dyrebare , som vi nu have seet af Johannes og Paulus . Om bi nu have en anden Smag , saa er det intet godt Tegn , da bor vi beklage dette for Gud og anraabe ham om det rette Sind og den rette Smag . Vi maa ogsaa vide og bctcenke , at der er ingen Synd paa Jorden saa sorskrcekkelig , ingen som maa opvcekke Guds Vrede saaledes som Utaknemmeligheden og Foragten for hans store Velgjerninger , og nu er dog ingm Guds Velgjerning storre end den , at han gav os sin egen Son og i ham det evige Liv , da vi havde fortjent at boere vore Synders rette Lon , og derfor . - Hvorledes skulle di undfly , om vi ille agte en saa stor Salighed ? ( Hbr . 2 , 3. ) . Og endnu mere om vi ogsaa engang have seet Guds Herlighed i Kristi Evangelium , ere ved Troen blevne Guds Born og have smagt , hvor listig Herren er , da var det jo en forskrækkelig Utaknemmelighed, om vi nu holdt dette Evangelium for en ringe Ting , som ikke smagte os . Det var jo , hvad Jesus kalder at have „ forladt den fsrfte Kjwrligheo . " Paulus skriver ogsaa om den forste <b>Kjcerlighed</b> iblandt de Galater , at de da havde skattet Evangelium saa hoit , at de havde annammet ham , der vrcedilede det , „ som en Guds Engel , ja som Jesus Kristus <b>selv</b> " , og han tillægger: Hvor prisede I Eder da salige ! Thi jeg giver Eder det Vidnesbyrd , at om det havde vwr ' et muligt , havde I udrevet Eders Vine og givet mig . Saa hoit vurderede de Evangelium dengang . Men da de vare blevne „ fortryllede " af en „ fremmed Lcerdom " , saa at de ikke adlod <b>Sandheden</b> , men vilde blive retfærdige ved Loven , saa kalder han dette , at de „ korsfæstede Kristus vllany " ( Gal . 3 , 1. ) . Og det er jo forskrækkeligt , at saadant ifolge Kristi egne Ord kan stee , eller den forste Kærlighed forlades , ' medens man dog vedbliver at arbeide og lide for Kristi Navns Skyld , er varsom mod falske Aander og har skarpt <b>Blik</b> til at kjende disse , og altsaa ved en stille indre Hcnsygnen fra Omvendelsens og Troens rette Ovelse ; thi saaledes lyde Kristi betænkelige Ord i Aab . 2 , I — s : Jeg veed Dine Gjerninger og Dit Arbeide og Din Taalmodighed , og at Du ilke lan fordrage de onde ; sgDu provede dem , som sige sig <b>selv</b> at vwre Apostle og ere det ilke , og har befundet dem at vwreLsgnere ; og Du har udftaaet og har Taalmodighed , og Du har arbeidet for mit Navns Slyld og er ilte bleven trar . Men jeg har dette imod Dig , at Du har forladt Din forfte <b>Kjwrlighed</b> . Kom derfor ihu , hvorfra Du er falden , og vend om og gjor de forrige Gjerninger ; men hvis itle da lommer jeg snart over Dig og bil flytte Din Lysestage fra dens Sted , hvis Du ilte omvender Dig . Endelig skulle vi tillige mcerke ( hvad der ogsaa kan sees af disse Kristi Ord ) at om Kristus altid stal blive os dyrebar og kostelig , vort <b>Hjertes</b> alt i alt , og den nye Sang aldrig blive os gammel eller lang , saa er det nodvendigt , ikke blot at den rette Ll ? re , men ogsaa det retsindige Liv bevares , at vi ere i bestandig

<h4>2119</h4>enoigt for at frelse hans <b>Hjertes</b> Stat , som Barnet jo er for Mderhjertet . Endelig tommer ogsaa hertil , at enten vi forståa ette eller ikke . have vi dog sect ham , siger Johannes i vor Text ; , an „ boede iblandt os , og vi saa hans Herlighed som den En « garnes af Faderen , fuld af Naadc og <b>Sandhed</b> . " Heri finde vi larsagen til . at Apostlene troede paa dette store Under med saa»an Vished , at de villigt led allchaande Marter og Pine for zans Skyld ; naar de vare blevne hudflettede for deres Bckjendelse lf hans Navn , gik de fra Raadet med Gloede over , at de vare icrrdige til at lide Forsmædelse for hans Skyld . At de vare saa nsse ' i deres Sag . det kom deraf , at de havde seet hans Herligled. Om vi ogsaa havde vcrret noervccrende og seet hans Herlighed, vare vi vel ogsaa mere troende , end vi ere . Men hvad mener Johannes , naar han siger i vor Text : . Vi saa hans Herlighed , " da jo dog Guds Son skulde os algode voere i den dybeste Fornedrelse her paa Jorden ? Med . ians Herlighed menes da alle de Tegn paa hans Guddoms Majestcet , som efter Guds Raad aabenbaredes nu og da for at styrke vor svage Tro . Til disse Tegn kunne vi forst regne den himmelske Hcer ' stares og alle Englenes Tilbedelse , som ogsaa Kristus <b>selv</b> sagde : „ Herefter stulle I see Himlen aaben og Guds Engle at såre op og ned over Menneskenes Son , " ligesom en Konges Herlighed ofte vises ved al den Pragt og Glans , som udmoerker hans Omgivelser . Da dette Barn var fsdt i Bethlehems Staldtrybbe, da opfyldtes hele dens Egn i den Nat med et himmelsk Skin , og utallige himmelske Vccsener talede og sang om dette Barn for dem , som vaagede der . Og alt , hvad den fattige <b>Moder</b> og Fosterfaderen stuldc gjore med dette Barn baade nu og ved Flugten til Mgyoten , det blev dem tilkendegivet af Himlens Engle . Senere see vi ogsaa , at da Jesus havde gjenncmgaaet Fristelserne iOrken , ! om Englene og tjente ham . Og endnu senere , da han var opstanden , vare Engle ved Graven de forste Forkyndere af hans Opstandelse . Saaledes sandedes det allerede yer paa Jorden , hvad den himmelske Fader tidlig havde forudsagt: „ Og alle Guds Engle stulle tilbede ham " ( Hbr . 1 , 6. ) . Ja tamk blot paa det Ord ! Hvad er det for et Boesen , om hvem Gud <b>selv</b> , som har forbudt al Tilbedelse af stabtc Vcesener , siger : « Og alle Guds Engle stulle tilbede ham ! " Og saadant havde Johannes seet med sine <b>Oine</b> . Dette er da vist noget til Forklaring af det Ord : „ Vi saa hans Herlighed . " Hertil kommer mdere alle de Almagtsgjcrninger , som Kristus nu og da gjorde , saasom den allerede noeunte , da han i Orken skabte Brod for 5000 Moend , eller da han ved Lasarus ' Grav opvakte den dode , som allerede stinkede af Forraadnelse , eller med Magt bod de onde ' Aander at fare ud af Menneskene . Men scerstilt synes Apostlene at have kaldt det hans Herlighed , hvad der skete paa Forklarelsens Bjerg , da de med et saa deres Herre at straalc i et himmelsk Lys . hans Klceder bleve hvide som Sne , to af Himlens Indvaanere

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100603020'target=_>, 1850, Det Nye Testament med Forklaringer, Indledninger, en Harmoni for de fire Evangelier, en Tidstavle for Apostelhistorien og et Sag-Register</a><h4>1686</h4>Johannes , Son af Zebedceus og Satome , Broder as lacodus , var fodt i Bethsaida , og Fister tigesom sin Fader , forinden Jesus katdte ham til sin Efterftlgelse. Hans <b>Moder</b> synes at have varet en from <b>Qvinde</b> , thi vi finde bende blandt dem , som med udmcerket <b>Kjcerlighed</b> og Trostab holdt sig til Jesus Matth . 27 , 56. Mrc . 16 , 1. Da Johannes den Dober optraadte , sluttede Johannes sig til ham , indtil han , bevceget ved sin Mesters Vidnesbyrd , sogte Jesu Selstab ; og fremfor alle andre Disciple stod han tilligemcd lacobus og Petrus meget snart ' i det fortroligste Forhold til Forloseren Lue . 8 , 51. Mattb . 17 , 1. Cap . 26 , 37. Joh . 13 , 25. Cap . 18 , 15. 19 , 27. 20 , 2. 21 , 20. Grunden til dette Privatvenssab med Jesus laa i Eiendommeligheten i Johannes ' acmdelige Natur , thi han var af et mildt , modtageligt og hengivende Sindclag , boielig og let at bevcege , fuld af dyb Folelse og tevende indre ' Anskuelse . Hellig Mildhed , Sagtmodighed , Mmygbcd og Kjartighed titkjendegiver sig i hans Karakter , et elskende <b>Hjertes</b> suldkolnne Hengivelse for Frelseren og for Brodrene , en brandende Iver , som omfatter den hele ' Verden , og langt fra al umandig Svaghed . Dette i <b>Sandbed</b> gudelige Sindclag har ved hans Forbindelse med Forloseren udviklet sig i ham iidaf hans naturtige Anlceg , thi bans naturlige Sindsbestaffenhed var ingenlunde pletfti Mrc . 10 , 35 flg . Lue . 9 , 49. 54. Efter Herrens Ophoielse finde vi Johannes i Jerusalem Avsthst . 1 , 13. 3 , 1 flg . 4 , 19. , og han gjaldt for en Ststte i Kirken Gal . 2 , 9 ; ifolge Sagnet stal han vare forbleven i Jerusalem indtit Marias Dod , som indtraf i Aaret 48 efter Christi Fodsel . Han synes at have belivet, sig til Ephesus i Lilleasien strår efter Pautus ' Dod , og der , under Landsforvisningen til Pathmos , at have strevet Aabenbaringen . Om han fra den Tid mfbrudt er forbleven i hin Egn , er uvist , men det er udenfor al Tvivl , at han i sildigere Tid har levet i Ephesus og virket der til henimod Slutningen af det forste Aarhundrede . Han opnaaede en hoiere Alder end de andre Apostte , og overlevede dem alle ; at Keiser Domitian tod ham kaste i logende Olje , uden at han deraf led nogen Skade , er en ikke titstrakkelig stadfcestet Efterretning . Johannes ' aandetige Eiendommelighet » synes at have bavt en saadan Ind- Melse paa hans Virksomhed , at hans Bestrcebetser mere vare rettede paa Lbristendommcns Forhoielse , Renselse og Forcedting der , hvor den allerede var , end paa dens Grundtceggetse og Udbredelse . Det klareste Speil for hans Aand og <b>Hjerte</b> ere hans Skrifter og i Sardelesbed Evangetiet , hvilket en from Theolog bar kaldt Christi <b>Hjerte</b> . Johannes ' Skrivemaade er opboiet og ukunsttet , hvorfor de Gamte gave ham Drnen tit Sindbillede ; i bans Fremstilling hersker hellig Rotighed og Sindighet » , og med helligt Alvor er bellig Mildhed forenet og dyb inderlig <b>Kjcerlighed</b> . ^ Han isar giver os et himmetst-acmdeligt Billede af Forlostren, og hans ' eiendommelige Aandsbestaffenhed gjorde ham stikket til at fremstille Lhristendommens Dybbeder ' i Folelse og Lcere ( <b>sand</b> Mvstik og agte Gnosis ) . Evangeliet er hoist sandsyntigen strevet i Ephesus og vist ester Jerusalems Ddelceggelse , omtrent ved Aaret 80. Da der i og omkring Ephesus fcmdtes , foruden de Modstandere af Christendommen , som Apostethistorien og Partti Brev til Epheserne tare os at kjende , ogsaa andre , som sogte at forene deres osterlcmdste, hedenste Philosophi med Christustaren , og derved forfatstcde og fordarvede denne : saa tog Johannes ved sit Evangetiums ' Forfattetse vistnok Hensyn paa disse Modstandere , men Evangeliets HovedLttemed bliver dog altid det . som han <b>selv</b> angiver Cap . 20 , 31. , at stildre Tfesus som Christus , og at indbyde Alle til Troen paa ham og til Saligheden i hans Forening . Johannes forer Beviset for Jesu Messiasvcerdighed hovedsagetigen : ved Johannes den Dobers Vidnesbyrd , som med futd Vagt gjsres gjceldende ; ved de af Herren udfsrte Mirakler, hvilke han anser som Aabenbaringer af hans Hertighed , og endetig ved det Gmt . Testaments messianske Spaadomme , som ere gangne i Oftfytdetse Paa Jesus . Da det fornemmetigen var Johannes om at gjore , at godtgjore og bevise Forlsserens guddommetige Persontighed og hans guddommetige Sendetse , og at han er den eneste Magler mellem Gud og Menneskene , af hvis guddommelige Livsfylde alene Menneskene kunne ose guddommetigt Liv , og ved hvem åtene de kunne komme i Forening med Gud : saa meddeter han desaarsag isar saadanne Jesu Taler og Udsagn , som angaa hans Persons Vcerdighed , Betydning , Vigtighed og Uundvcerlighed, og just denne Gjenstand udgjor ogsaa Middelpunktet i Striden mellem Forloseren og hans Modstandere . Jesus er Guds enbaarne Son , det Kjod-blevne

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014110426005'target=_>Francke, August Hermann, 1847, Huus-Postil, eller Prædikener paa alle Søn- og Festdage i Aaret</a><h4>642</h4>Betragter jeg ellers vor Text , og betcenker , hvem der vel paa det ommcste har elsket vor Herre Jesus , saa var det udentvivl hans Moders Maricr omme <b>Hjerte</b> . O hvad maa der have gaaet for sig ! hvilke Kærligheds-Flammer maaeder ikke have voeret tildette Barn , som hun havde fsdt , og om hvilket der var sagt hende saa store Ting ! Altsaa sige vi da hellerikke uden Grund , at Kærligheden til den Herre lesum er allerommest hos en saadan Sjoel , der aandeligviis er bleven Christi <b>Moder</b> , saaledes , og i den Forstand, som Christus <b>selv</b> taler derom ( Matth . 12 , 49. 50. ) . Men for Kortheds Skyld ville vi for denne Gang ikke tale mere derom, helst siden det ogsaa vil blive klart af det Folgende . 2 ) Vi have nu for det Andet ogsaa at betragte : Hvorledes denne <b>Kjcrrlighed</b> til den Herre lesum viser sig . Om Joseph og Maria hedder det i Texten : " De forundrede sig paa de Ting , som bleve sagte om ham . " Seer , der viser sig den forste <b>Kjcrrlighed</b> til den Herre lesum i denne Forundring. Der er vel mangestags Forundring til , idet den kan saavel komme af Vankundighed og Vantro , som af en af Gud virket Crkjendclse og en <b>sand</b> Tro . Men hos Joseph og Maria

<h4>1626</h4>Nu er vel dette vistnok noget stort , at Maria har vcrret en <b>Moder</b> til Verdens Frelser ; men Sagen <b>selv</b> , som her forkyndes , nemlig , at Gud vilde give Verden en lesum ' eller Frelser , som skulde forlsse det menneskelige Kjem , og at denne Frelser vel skulde vcere et <b>sandt</b> Menneskes Barn , men ikke noget blot Menneske , men Guds Son , og oprette et saadant Kongerige , som ingen Ende stulde have , det er Guds Rige , eller gjore os evig salige : denne Sag <b>selv</b> , siger jeg , var endnu meget storre . Og denne vedkommer ligesaavel os som Maria ; hvorfor vi her maae rette vort ganste <b>Hjerte</b> , Sind og Gemyt paa den i disse Ord aabenbarede og usigelige Guds <b>Kjcerlighed</b> , eftertcenke samme , undre os derover , annamme den med AZrbodighcd og Taknemmelighed , og herefter ganste overgive og nedscenke os i samme ; thi den Saliggjorelse , som her forkyndes , er vor ; hvad denne Marice Son er , det er han for os ; vi hore til hans evige Kongerige , ja vi stulle vcere Konger , og til evig Tid herske med ham , saaledcs som Skriften paa

<h4>3864</h4>ham , blloe hanS sande Disciple , og deelagtige i hans Aand ; thl hvad for det fsrste Loven angaaer , saa smage de sammes Bitterhed i Forstningen af deres <b>Hjertes</b> sande Omvendelse , naar deres arvelige og gjorlige Synders dybe Fordærvelse i fllg Omvendelse ret bliver dem opklaret og givet tilkjende , saa de nu see og fole , hvorledes de hidindtil have ligget under Lovens Forbandelse, staaet i et Fiendstab mod Gud , ikke voeret hans Lov underdanige, og heller ikke formaaet stigt , og derover ere stedse blevne under Guds Vrede og under Dsdens og den evige Fordummelses Dom ; men siden , naar nu denne Angest har drevet dem til Ehristum , der har forloft dem fra Lovens Forbandelse , og de ikke alene have faaet Syndernes Forladelse ved Troen paa ham , men endog annammet den Hellig-Aand af ham , ved hvilken Gud nu har givet sin Lov inden i dem , og skrevet den i deres <b>Hjerte</b> , saa smage og erkjende de Loven endog paa en liflig Maade , idet de da ikke alene vide , hvilken Kjoernen er af samme , nemlig , at elske Gud over Alting , og sin Nccste som sig <b>selv</b> , men deres <b>Hjerte</b> , Sind og Gemyt er da ogsaa isandhed opfyldt med <b>Kjcerlighed</b> til Gud og deres Ncrsie , og.omendfljont Synden hcenger altid ved dem , saa lade de den dog ikke herske over sig , men stride imod den ved Christi Kraft , overvinde den , og aftage daglig i det Onde , men tiltage i det Gode , og jo mere og mere fornyes til Guds Billede . Men hvad Kjcrrnen af Evangelium angaaer , saa erkjende og smage ogsaa de aandeligsindcde Mennesker den ; thi Dcekket er borttaget af deres Hjerter , saa de nu see ind i det Inderste , erkjende den Naade og <b>Sandhed</b> , som er bleven ved lesum Christum, med Troens <b>Oine</b> , ste hans Herlighed , som den Ecnbaarnes af Faderen , antage ham i den sande Tro , toe og rense sig i hans Blod , og mode for Guds Ansigt , som bestcrnkte og toede med hans Blod , faae af hans Fylde Naade for Naade , og isandhed troste sig derved , at Herren har sagt til ham , deres Herre : „ Scet dig hos min hsire Haand , indtil jeg lcegger dine Fiender til dine Fodders Fodskammel ; " thi saa stride de og i hans Kraft , og troe paa det allervisseste , at alle deres Fiender , Synden , deres fornemste Fiende , med alt sit Anhang ere lagte under deres Herres Fodder , men de stulle komme til at triumphere og regjere med ham i evig Glorie og Herlighed . Dette er Altsammen Kraft og <b>Sandhed</b> hos et aandeligt Menneste , der erkjender dette af Evangelium i den Hellig / Aands Lys .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010021603013'target=_>Arndt, Johann, 1855, Johan Arndt's Postille, eller Aandrige Forklaringer over de aarlige Søn- og Helligdages evangeliske Texter</a><h4>30</h4>v Naadr ^ ligesom ogsaa denne Guds Naade er langt hsiere og bedre , end alt , hvad man kan suste , bede og begjcrre , ja , denne uudsigcligc Guds Naade mcrttcr saalcdcs det troende <b>Hjerte</b> , at man ei snsker mere , ja , at man derimot » holder Himmel og Jord for intet , ligesom den kjerre Asaph i Ps . 73 , 25. siger i Fornemmelsen as denne Naadcns Kraft :  Hvem haver jeg i Himlene ? og lige ved dig haver jeg ikke Lyst til noget paa Jorden , " ja spsraer intet efter mit Legem og Liv ;  thi din Mistundhcd er bedre end Livet , " Ps . 63 , 4. Teraf er de Helliges Dnske udssydt :  Lad mig kuns finde Naade for dine Dine , " af den Aarsag , at samme begribcr alt i sig , og er alt. Det fjerde Houcdpunct i den sande Religion er Troens rette Forstand i den Artikcl om Rctfcrrdiggjsrelsen for Gud , nemlig , at den sande , levende og saliggjorcnde . Troe ene og alene grunder sig paa Guds Naade og Barmhjertighed i Christo , og at den maa verre rcen og übesmittet ; thi saa snart en eneste , ja den allerringeste Tillid paa Menneskcvcrrk , Egcncrre , egen kjedelig Kjcrrlighcd . Vcrrdighcd , Hellighed , Rctn ? rdighed , Fromhed og Duclighed bliver indblandct med. saa er Troen ei rccn og übesmittet , ja , det er en falsk Troe ; thi saa snart Troen hcrfter ved noget andet , end ved Guds blotte Naade i Christo , saa er det ingen ret Troe , og er afrevet fra Naaden, ja Naaden er da forkastet , saa er da ingen Syndernes Forladelsc af Naade der , heller ingen ret bestandig Trsst , heller ingen ret , rcen og <b>sand</b> Guds Kjcrrlighcd , men Egenkjærlighet » . Saa er der heller intct ret Haab ; thi Haabcts Art og Egcnssab er , at det venter alt for Christi Skyld , ei for egen Vcrrdighcds Skyld : saa er heller ingen Guds Wrc der , thi da giver Mennesket Gud ikke alene Wrcn , men sig <b>sclv</b> ; saa er heller ingen Ncrstcns <b>Kjcrrlighed</b> der , hvis Art er , at den gjor alt omsonst , uden Hensyn til Vclonning . mm alt af rcen Guds Kjcrrlighcd , af ydmyg Takncmmclighed for den store Guds Naade , som af Evangeliets Forklaring paa Ssndag Scptuagcsima og paa mange andre Steder noksom er at sec . Thi hcrnger disse ved hinandcn : Guds Naade . Christus , den Helligaand , Troen , Syndernes Forladelse . <b>sand</b> Trost , Guds Kjcrrlighcd , bcstandigt Haab . Guds Wre , Ncrstens Kjcrrlighcd . Wmyghcd og Takncmmelighed. Naar nu Troen bliver saa rcen . saa er den som et klart Speil , i hvilket man ingen Plct seer . men alene Ehristum og Guds Naade . Saa er den det arme , elendige Barn , som alene Guds Barmhjertighet » bedcrkkcr , grcrder og sukker alene efter Guds Naade , ligesom et lidct Barn ester sin <b>Moder</b> : saa siger Herren :  Mine

<h4>368</h4>rette Hemmeligheder , som her vare os skjulte , blive aabenbarrde i den hoieste Fuldkommenhed , thi hisset vil alt verre fuldkommrnt , hvad som ber h.ar vcrret Stykkevcrrk . Derfor kaldes den ogsaa en Gjenoprettrlses-Dag , da alle Ting stal gjenoprcttes , lii « 8 l-oslillilione » omnii , , » . Vore ny forklarede Legemer stulle jo ogsaa da have deres . Liflighed og Fornyelse i himmrlste Creaturer i Paradiis . Nu kunne vi vel ikke see , hvorledes vi have vort Liv af Liv ' sens Troe , som er Christus , om hvem det hr dder . '  I ham var Livet og Livet er Menneskets Lys , " Joh . 1 , 4 , ; men da stulle vi faae at see , lworrdes alle Ting have Livet af ham ; thi han er Livets Ord , hvorved Alt er stabt , og han vil med sin Klarhed oplyse Himmel og Jord , eftersom samme Klarhet , vil stinne fuldkommen og ei stykkrviis . Hvad er Sommerens Mdelighed ? Deiligt Solskin , blomstrende Tråer og Fuglenes Sang ; men da vil Christus verre vor Sol , vort Paradis , og Englene med alle Udvalgte ville love Gud . 4. 3 a vil Guds Ni gr b cg ynde . Vi bede jo hver Dag om Guds stiges Komme , og her have vi Naadrns Nige , men hisset vil blive Guds Herligheds Nige .  Guds Nige er , Netfcrrdighcd , Fred og Glcrde i den Helligaand " , hedder det i Norn . 14 , 17. Dette Nige brgynder vel her men hls , et vil det ful ^ kommeligen blive aabenbarrt i fuldkommrn Netfcrrdighrd og Hellighed , fuldkommen Fred , fuldkommen Glcrde , som Ingen vil tåge fra os , fuldkommrn Erkjendelse af Gud og den hellige Trefold ighed fuldkommen Erkjendelse af Christus , hans Person , bansNaade-Gjerninger hans Kj.rrllghed og Trofasthed , saml Beskuelse af hans Herlighed hos Guds hsire Haand . Her have vi vel Troens fuldkomne Netfssrdighed , fordi vi have lfort os ( shristum ; men hisset stulle vi have Livets suldkomne Nrtftrrdighed og Hellighed , saa at det ganske Guds Herligheds Nige stal da fuldkommeliq stinne l os med en evig fuldkommen Netfcrrdigbed , Herlighed , Fred Gl ^ dc Wiisdom og Erkjendelse , saa at intet <b>Oie</b> har ' , ' eet saa stjont / intet Sre hort saa lifllgt , og ei i noget Menneskes <b>Hjerte</b> opsteget saadan Glcrdr ; thi hvad kan vel verre mere yndefuldt , bvad , n.re skjont , hvad saa hcrligt som Gud srlv ? Det maa jo v ^ rre det lifligste , meest frydcfulde og heieste V.rsen hvorl Gud boer ; thi der vil vi faa at s.e , hvorledes Klarhed og H.rliqhed dll opfplde alle Yngle og alle Mennesker , og hvorledes Guds Herligbed vil stinne i dem , da Gud vil vare Alt i Alle . Der vil alle verre tilfreds ; Alt , . bvad en Udvalgt vil rnske . det vil han alt have i hoieste Grad . tbi Gud vil <b>selv</b> verre Alt ; der vil den nye Himmel verre fuld af Gl.rde ' fuld af Liv . fuld af Fred , fuld af <b>Kjcrrlighed</b> , fuld af Rctfcrrdighed , ja der vil v. n-e N ' ldko , msen <b>Kjcrrlighed</b> . Hvor kj ^ rrligt vil dog Guds Udvalgte have det med lwerandre . hv « r venligt vil de tale nied hinanden om Guds Almaat Guds Barmhjenigbed . Viisdom og <b>Sandhed</b> , oa fordi Alt er ucndeliqt vil

<h4>3612</h4>Synd Vreden er . Vi tale ikke her om Embeds-Vrede hos Prcrdikantere , Ovrighed , Fader og <b>Moder</b> imod Udyder ; thi da maa man vredes og kan ogsaa uden Synd vredes , ligesom Ps . 4 , 5. siger :  Bliver vrede , og synder ikke . " Moses vrededes over Afguderiet med den gyldne Kalv ( 2 Mos . 32 , 19. ) , Pinehas over Utugt ( 4 Mos . 25 , 7 ) , David over falsk Lcrre ( Ps . 139,21 . : c . ) . Eli blev straffet , fordi han ikke engang saae suurt til sine Sonner , 1 Sam . 2 , 24. Men vi tale her om den fiendtlige , hcevngjerrige , misundelige , djcrvelste Vrede . Saadanne Folk handle og tale vel udvortes venligen med deres Nssste ; men i Hjertet ere de Mordere formedelst Vrede og Misundelse , og forglemme Gud i Himmelen , at han er vores alles Fader , ja de forncrgte Gud i Himmelen ; de forglemme , ja forn ^ gte Guds Son , at han kalder os alle sine Bredre og har forsonet os med Gud formedelst sit Blod ; de forglemme , ja forncrgte den Helligaand , at han er en <b>Kjcrrlighedens</b> Aand , som Gud udgyder over os ligesom en himmelsk Balsom, hvoraf vi skulle blive milde og sagtmodige ; de forglemme og forncegte , at Menneskene ere skabte af Gud til hans Billedes Liighed , som bestaaer i Sagtmodighed og Barmhjertighet , og gjore sig <b>selv</b> lige den lede Djaevrl , som har et uforsonligt Had og er en afsagt Guds og Menneskets Fiende , ja en Morder fra Begyndelsen . Saaledes gjor Vreden Menneskene ogsaa til Mordere . Bernhardus siger :  Vreden er en uhyre Drage , hvis udspyende Ild drcrber , hvad den kuns angriber . See , hvad hjcrlper det dig nu , at du stiller dig udvortes venlig , og har dog en saadan gruelig Morder i Hjertet ? Saaledes gjorde Pharisceerne ogsaa , hvorfor Herren raaber Vee over dem : " Vee eder , I Skriftkloge og Phariscrer , I Oienskalke ! " Matth . 23 , 27. Vi have ei noget at gjsre med de gamle Phariscrere , men med de nye Phariscrere og Hyklere . Tildeels bryder ogsaa Vreden ud i udvortes Gebcrrder , siaaer som en gloende Flamme ud i begge <b>Oine</b> og gjor Mennesket grueligt og afskyeligt , som vilde han med sine Gine drcrbe sin Ncrste . Det hedder " Naka , " ligesom vi og see hos Cain , der , da han ncrrede den mordiske Vrede i sit <b>Hjerte</b> imod sin Broder , forstillede sit Ansigt , saa at han saae ganske fcel og grum ud , hvorfor ogsaa Gud sagde til ham : " Hvi er du vred , og hvi er dit Ansigt falmet ? " 1 Mos . 4 , 6. Deraf lcrre vi , at baade Dyd og Udyd lyse Menneskene ud af Oinene ; Hi saadant et <b>Hjerte</b> , som et Menneske har , en saadan Skikkelse har han og .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008082512001'target=_>Luther, Martin, 1863, Dr. Martin Luthers Udførlige Udlæggelse af første Mosebog</a><h4>2391</h4>213. Philosopherne ' have ogsaa hacrrdt cmgrebet og revset Mistanken som en giftig og skadelig Ting i enhver god Forstaaelse . Herom taler Cicero mesterlig i sin Bog I.Nliuß , og revser haardt det bekjendte Sprog af vcer Ven paa den Maade , at dn ogsaa kunde vcere Fiende . Og Cicero har Ret , nåar han siger , at der ikke kan vcere noget bestandig ! Venskab der , hvor man giver Mistanken Rum . Men nu er det Mistanke at stutte , at man engcmg vil komme til at hade den , som man nu elster . Og dette er ikke saa uligt Ord , at <b>Sandheden</b> er <b>Kjcerlighedens</b> <b>Moder</b> . Thi hvor den Ene ikke holder den Anden Tro og Love , der kan hverken det huslige eller borgerlige Liv beståa . Thi hvor megen Elendighed anretter ikke Mistanke blcmdt Wgtefolk , <b>selv</b> for ringe Aarsagers Skyld ? Men nu folger der langt Drre Ulykke deraf , nåar denne Syge treenger ind i det almindelige Liv . 214. Og dog lcerer Abraham os med sit Exempel at ncere Mistanke; ja , hvad der er endnu mere , den hellige Skrift befaler og tilraader os at vcere mistcenkelige , ' idet den rent ud siger , at alle Menneske ere Lsgnere , Ps . 116 , 11. En Mands Fiender , siger Micha 7 , 6. , ere hans Husfolk . Fortro dig ikte snart til en Ven , Sir . 6 , 7. Fortaber Eder Ne paa Fnrster , Ps . 146 , 3. Våer din Munds Dore for hende , som ligger i dit Skjod , Micha 7 , 5. Og Christus , siger Matth . 10 , 34. : jeg er Me kommen at sende Fred , men Svcero . 215. Dette er klare Sprog af Skriften ; men alligevel betcenke Enhver , som vilde folge dem , hvad for et Liv vi vilde fore , om vi stnlde leve i saadan Mistro , Mistillid og onde Tanker om hinanden indbyrdes , Mand med <b>Hustru</b> , Undersaatter med Vvrighed , Tjenestefolk med sin Husbonde 0. s . v. Og hvor bliver der saa af Christi Bud : dmmer Me ; se Me Stjoeven i din Broders <b>Die</b> , Matth . 7 , 1. 3. ? Hvorledes vilde det da ogsaa gaa med Kirken , som jo foreholder Synderne Boden og lcerer os , at vi ikke ganske maa forsage for Syndernes Skyld ? Derfor gjcelder jo ikke i Kirken luristernes almindelige Regel : hvo der engang har begaaet en Ugjerning , af ham maa man vente sig , at han altid gjor Ondt , efterdi Christus ogsaa vil , at man skal forlade en Broder hans Feil , sytti Gange syv Gange . Hvorledes ville vi da nu forene og forlige med hinanden faadanne Sprog , som ganske modsige hinanden ? 216. Hvad angaaer hint Sprog af Bias , fom Cicero cmgriber faa heftig , da er det min fulde og faste Tro , at Cicero aldrig har forstaaet , hvad Bias har ment dermed . Thi Bias ' s Scetning gjcelder ikke übetmget, men betinget ; det er , det er ikke Meningen , at En , som har en Ven kjcer , hos sig <b>selv</b> stal gjore sig de Tanker , at han atter vil blive ham fiendtlig ; men at det vel er muligt , at hau sioen vil kunne hade den , som han nu elster . Ligesom ogsaa Augustinus siger , at det i Livet ofte gaaer saa , at man kommer til at elske En , som man forhen hadede . Men dette philosophiste Sporgsmaal ville vi overlade Philosopherne at underfoge , og ville betragte denne theologiste Sag , nemlig hvad Abraham gjorde , og hvad Skriften forbyder at gjore . Nu forbyder

<h4>2408</h4>og beholde , indtil den Hellig-Aand forbedrer den . Derfor stal man ikle forlade sig paa et Menneske , forsaavidt han er menneskelig , det er sirsbelig, <b>selv</b> om du , hvad angaaer den ydre Vandel , intet Daddelvcerdigt ser paa ham ; og dog stal du heller ikke vcere fiendtlig mod ham , saaledes at du , hvorvel du anser ham for ond , dog derfor Me forlader ham eller mistvivler om hans Forbedring og Omvendelse . 239. Men om Enhvers scerlige Kald og Embede , hvori det ser ud , som om der handledes mod <b>Kjcerligheden</b> , tale vi her ikke . Saaledes ser jo Riset og Straffen , hvormed en Husfader tugter siue Born eller Tjenestefolk , ikke ud som <b>Kjcerlighed</b> , og at Bvrigheoen straffer Misdcederne med Svcerdet og lader dem drcebe , ser ud , som om det var Vrede ; men de have Befaling til at gjore dette , og de vilde synde , om de vilde indrette sin Embedsforelse ester denne almindelige Regel , at man stal elske baade Fromme og Onde , hvilken Regel dog ogsaa de stulle vcere underkastede og lydige udenfor sit Kald . 231. Saaleoes stal man svare paa Propheten Michas Ord , nåar han siger Kap . 7 , 5. : bevar din Munds Dore for hende , som ligger i dit Stjod . Thi det vil sandelig Propheten ikke , at Wgtefolk indbyrdes stulle vcere mistcenkelige , hvilket aldrig lover af uden Tvedragt og Sorg , eller give Rum for Mistanke ; men han vil , at Manden stal elste sin <b>Hustru</b> , saaledes som St . Paulus stjont og kraftig udforer dette Bud i Eph . 5 , 25. 232. Og dette horer ogfaa blandt en Hustrues fornemste Dyder , at hendes Mands <b>Hjerte</b> tor forlade sig Pan hende , Ordsvr . 31 , il . , det er , at han bcerer venlig og hjertelig <b>Kjcerlighed</b> til hende , ikke venter sig noget Ondt af hende , men er ganske vis paa , at han atter elskes af hende , og at hun rigtig forestaaer hans Husholdning . 233. Derfor regner ogsaa Augustinus faare stjont tre Slags Goder eller Velgjerninger af ZGgtestabet , nemlig den indbyrdes Tillid , Bsrnene og Hemmeligheden . Thi hvor der ikke er Tillid , der ville aldrig Sjcelene rigtig forenes , der ville de aldrig i <b>Sandhed</b> elste hincmden . Men nu er der intet Skjsnnere i Verden , end <b>Kjcerlighed</b> og Enighed blandt " Som ogfaa Sirach siger Kap . 25 , 1. 2. , at tre Ting behage Gud og Mennester : Brodres Sllmorlra.tia . hed , og Naboers Venstllb , og <b>Kvinde</b> og Maud , som forliges vel med hverandre . 234. Saaledes vil ikke Propheten , at der blandt Wgtefolk stal vcere Mistanke , Had og Fiendstab , men den holesie <b>Kjcerlighed</b> og Venstab, hvilket ikke kan finde Sted , hvor den Ene ikke troer den Anden . Men dog vil han ogsaa , at denne Tillid stal have en Grcendse ; thi det kan hcende , at den slaaer Feil og skuffer . Thi en <b>Kvinde</b> er et Menneske, og om hun end frygter Gud og giver Agt paa Ordet , kan hun dog falde , efterdi hun har Satan til Fiende , fom altid lurer og efterstrceber, og Naturen i sig <b>selv</b> er svag , og underliden kan dit Haab slaa Feil for dig . 235. Har du nu i Forveien betcenkt dette hos dig felv , faa vil du desto villigere tilgive hende , vil ogfaa bedrpves desto mindre , om der hcendes noget Andet , end du havde ventet dig ; da vil <b>Kjcerligheden</b> forblive og Enigheden ikke forstyrres . Thi der hcendes da Intet , som du ikke i Forveien bavde tcenkt paa , og <b>Kjcerligheden</b> er saare villig og redebon til at forlade . At kunne gjore dette er vel en fjelden Gave ; dog stal du vide , at saascmdt du er en Christen , maa du gjM det .

<h4>3836</h4>kjcere <b>Moder</b> . Om nu en saadan Ulykke ilke skulde gaa Nogen til <b>Hjerte</b> han maatte vcere haardere end en Diamant . 242. Saa er dette ogsaa <b>sandt</b> , at Folk , som ere i hoieste MM forfcerdede og bestyrtfede , blive af et ganste fremmed Sind og <b>Hjerte</b> , faaat de baade gjore og sige Ting , som de sidenefter glemme . Thi et <b>Hjerte</b> , som er saa aldeles forfcerdet og overiles af saa stor Ulykke , ved ilke af sig <b>selv</b> . At derfor Loth sov hos sine Dottre , det solte han nden Tvivl , thi et saadant Samleie ryfter hele Legemet og gjor stcerkt Indtryk baade vaa Legeme og Sjcel . Hvorfor siger da Moses , at Loth Me blev det våer ? 243. Svar : fordi han derved vilde vise , i hvor dyv Vedrovelse og Bestyrtelse Loth var , hvilken havde indtaget ham saa aldeles , at han ikte vidste , hvad han gjorde . Thi dette erfare vi i ganste übetydelige Ting hos tungsindige og forlibte Mennester , hvis Folelser og Tanter ere heftige , faaat de sige og gjore Meget , som de snart igjen forglemme . Thi Forelstede ere taabelige , og , som Digteren siger , de indbilde sig snart dette , snart hint . Derfor er det heller ilke nyt , at En , der er i heftig Beftyrtelse og ncesten ikke ved af sig felv , gjor Noget , som han sidenefter ikke lcengere huster . Derfor ere loderne daarlige Dialektikere ; thi Skriften siger ikke , at Loth ikke mcerkede dette Samleie ; men den siger det , at han stod op og ikke vidste , at han havde besovet sine Dottre . Thi Moses beskriver Loths store og hote Bedrovelse og ikke , saaledes som de urene Svin loderne tale om , Kjodets Lyst og Drifter . 244. Og dette tjener ogsaa til denne hellige Mands Undstyldning , at han ikke alene var i hmste BedrMelse , men ogsaa var bleven gjort bestjcenket af sine DMre , saaledes at altsaa to übeleilige Ting trceffe sammen , nemlig den hoieste Forvirring under den store Sorg , og Drukkenskab, som lettelig kan stade en Bedrpvet og Bekymret . Hvad stort Under er det da , om han om Morgenen ikke har yidst , hvad han om Natten havde gjort ? En Drukken taler ofte , og mener , at han taler saare vel ; men nåar han har sovet ud , erindrer han dog ikke , hvad han sagde . 245. Dette er mine Tanker om denne Tert . Thi jeg kan ikke finde mig i , at hine Hunde foregive , at Mofes skriver umulige Ting . Det hamder jo ogsaa i ringe Lidenstaber og Sindsbevcegelser , saasom i Tungsindighed eller <b>Kjcerlighed</b> , at man ikke altid kommer ihu , hvad man har gjort ; og det er ingenlunde umuligt , men er saare almindeligt og ster daglig , at et Menneske kommer saa dybt i <b>Kjcerlighed</b> , Had , Sorg , Glcede o . s . v. , at han enten af Vrede eller af Glcede ikke ved , hvad han siger eller gjor , medens han dog godt foler , at han siger eller gjor Noget . 246. Men at Loth derfor ganste kan undstyldes , det siger jeg ikke , og dette horer ikke hid . Det alene siger jeg , at det ikke er noget Umuligt, hvad Terten her siger om Loth . Thi han var beruset vaa to Maader : Legemet var fuldt af Vin , og hans <b>Hjerte</b> var fuldt as Sorger ; derfor er det intet Under , at han , da han bliver cedru , ikke ved , hvad han har gjort under sin Rus . Og dette tjener ogsaa til at lcere os , at man stal betragte den hellige Skrifts Fortcellinger med stsrre Omhu , og ikte fare los vaa den fom Svin , ovhidfede enten af Had eller af Vegjcerlighed . 247. Lyra forstaar ikke Texten rettelig , siden han beflylder den hellige Fader Loth for en faa stor Synd ; thi han har siddet i sit Kloster

<h4>3874</h4>Land er ufrugtbart ; Tornerne og Tidsterne kvcele Sceden , nåar den er spiret , og Satan efterstrceber ogsaa hemmelig de trygge Hjerter . Derfor stulle vi lade det vcere os nok , at der dog endnu er Nogle , som hore Ordet med Frugt og antage det . Men nn ville vi betragte selve Historien . . 15. Abraham frygter for , at han kunde komme i Livsfare ; derfor opdigter han en Logn i dette nve Herberge , og kalder Sara sin Soster . Men hvorfor gjor han det ? Fordi Gud derved vil lcere os , at ogsaa de allerhelligste Fcedre havde saadanne Skrobeligheder og Tilboieligheder som andre Mennester , paadet vi ikke skulle domme saaledes om dem , som vi dog pleie at gjore , at de vare Sten eller Stok , som ikke folte Frugt , Bekymring , Vrede eller Glcede , men vare ligesom Engle . Men det er ikke <b>sandt</b> ; en Engel frygter ikke , Abraham derimod er her fuld afFrygt og Bekymring for , at han kunde miste Livet . Derfor beskriver Moses ham som et Menneske og gjor dette til vor Trost . 16. Thi da han ikke har noget bestemt Ord fra Himlen af om , at hans <b>Hustru</b> stal bevares for ham og forblive ukrcenket , gjor han , hvad hans Fornuft raader ham til , i det Haab , at han tilligemed sin <b>Hustru</b> vil bevares og bestyttes ved denne Logn . Thi den Hellig-Aand driver ikke altid de Hellige ; han lader dem gjore nogle Ting ester sin Villie og sit eget Behag . Da Elias drcebte Baals Prcester , blev han dreven af Guds Aand , 1 Kong . 18 , 40. ; men da man siden forkyndte ham , at lesabel var bleven fortornet paa ham , frygtede denne samme Elias for , at der kunde vederfares ham noget Ondt ; derfor undveg han til ørkenen , for derved at redde sit Liv , 1 Kong . 19 , 4. Dette gjorde han efter sin egen Villie ; thi han havde ikke nogen sceregen Befaling af Gud om at undvige ; Fornuften indgav ham dette , at han vilde vcere sikker og vel bevaret , om han vilde opholde sig hemmelig i ørkenen . Den samme Elias , som havde stcerkt Mod og stor Iver , da han drcebte Baalsprcesterne, frygter derfor her , er forstrcekket og mener , at han ikke kan vcere sikker i noget Sted i Israel . Og dette er skrevet os til Trost , idet vi ellers ikke vilde have andre Tcmker om de Hellige , end at de vare Stok eller Sten , uden nogen Folelse . 17. Derfor sige Paulus og Barnabas med Rette om sig <b>selv</b> Ap . Gj . 14 , 15. : vi ere Mennester , Uge Villaar undergivne med Eder . De ere ikke bange for at sige dette til de afgudiste Mennester ; thi de Hellige fores og lceres ikke altid af den Hellig-Aand , men have sine Begjceringer , Glcede og Bekymring som andre Mennester ; derfor omgaaes de ogsaa med almindelige Gjerninger , saa , vloie , bygge o . s . v. Til at udrette disse ydre Ting er Menneskenes Fornuft og Duelighed nok . Og hvorvel ogsaa de Ugudelige gjore saadanne Gjerninger , saa behage de dog den fromme Gud vel og ere ham tcekkelige formedelst den Tro , hvori de leve ; derfor gjore de heller intet Underligt , medmindre de scerlig drives og bevceges dertil af den Hellig-Aand , og det er dem befalet i visse og udtrykkelige Ord ; ellers forblive og leve de saaledes , at de have et <b>Hjerte</b> som andre Mennester og have almindelige Gjerninger , som de omgaaes med , ligesom andre Mennester . De omgaaes Me med synderlige Gjerninger, saaledes som Pavens Hellige , der scette sin Hellighed deri , at de have en anden Klcedning , boalene paa sceregne Steder og adstille sig fra andre Folk , men imidlertid forsomme Gudsfrygt og <b>Kjcerlighed</b> og ikke bryde sig derom , men ere Myggesiere og Kamelslugere , Matth . 23

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016051048082'target=_>, 1878, Huus-Postil, eller En Samling af forskjellige Forfatteres, meest Luthers, Franckes og Pontoppidans Betragtninger over Søn- og Festdages deels Høimesse-, deels Aftensangs-Texter : samlede til en Aargang, og udgivne i den Hensigt, at befordre og vedligeholde christelig Huus-Andagt</a><h4>791</h4>for deres <b>Oine</b> , endnu vare saa vantroe , i Marc . 8 , 17. , og over Israel klages der , at de bleve ved at synde , og ikke troede vaa Herrens underlige Gjerninger , Ps . 78 , 32. Hvorfor Herren siger til Moses : Hvorlcrnge stal dette Folk opirre mig ? og hvorlænge vil det ikke troe vaa mig ved alle de Tegn , som jeg har gjort midt iblandt det ? Jeg vil staae det med Pestilentse og udrydde det , 4 Moseb . 14 , 11.12 . Derfor bFr den troende Sjæl dcsmere og npiere agte vaa alle Herrens Veie og Gjerningcr, Frelse og Hjælp , at den derved maa vore i Troen , og ikke altid være som et uforfarent Varn , der er frygtagtigt , og som et vaklende Rpr , og derfor letteligen kan kuldkastes og omkomme . Nu , I kjære Sjæle ! agter vel vaa eder <b>selv</b> , I , som hpre eller Icese dette . I mcerke vel ofte Vantroe , Frygt og Tvivl i Eders Hjerter , Ord og Gjerninger , o ! seer vel til , om det ikke er en herskende ' Vantroe ? om I nogen Tid have i <b>Sandhed</b> med Disciplene annammet Iesum som Eders Frelser ? om I have efterfulgt ham i Lydighed ? om I have af <b>Kjærlighed</b> til ham fornegtct Eders Egenvillie og Veie , og om Eders Onste og Attråa er kun alene at blive ved ham , baade i gode og onde Dage , baade i Fare og Strid , baade i Npd og Dpd ? Er det ikke saa , saa anseer ikke Eders Frygt og Tvivl for en Skrøbelighed , og troster Eder ikke med Disciplenes Erempel og andre Helgenes , som og undertiden ere blevne fristede af Frygt og Tvivl . Det er da visseligen en herskende Vantroe hos Eder , og I maae fprst Icerc af ganske <b>Hjerte</b> at omvende Eder til Herren , og at antage Jesu Naade ien <b>sand</b> Troe . Men finde I i PrFven , at I i <b>Sandhed</b> have staaet i Troen , men paa en eller anden Maade , som Eder nu er viist , ere blevne forrastede , og ere komne ind i Vantroe , Frygt eller Tvivl , saa griber Sagen an , saasom nu er bleven viist , efter Disciplenes og andre Troendes Erempel . Agter det ikke for en ringe Ting , men for en saadan Feil og Synd , som Gud har stor Mishag udi , paadet der ikke stal blive en herskende Vantroe deraf , hvorved I snarligen kunde falde fra den levende Gud , Ebr . 3 , 12. For alting lærer ret i en <b>sand</b> Hjertens Idmygclse at bede og raabe til Herren , og vender Eder ikke andensteds hen . Og skulde det end synes , at Jesus sover i Eders Npd , og ei bekymrer sig om Eder , bliver dog ved , og lader ikke af , Herren redder dog omsider sine Udvalgte , som raabe til ham Dag og Nat , endog han er langmodig imod dem . Luc . 18 , 7.

<h4>2812</h4>de stulle hade Fader , <b>Moder</b> , Broder , Spster , <b>Hustru</b> , Gods , og tilmed sit eget Liv ; uden det kunde de ikke blive hans Disciple , Luc . 14 C - , de skulde scelge alt det de havde og Mge ham , Matth . 14 , 21. Her kunde nogen ville spørge : er dette een Mening , at elske Gud eller forncegte sig <b>selv</b> ? jeg svarer ja , saa meget du elsker Gud , saa meget forncegter du dig <b>selv</b> ; vist kan en eller anden tiltagen Selvfornægtelse af egne Forsætter og Indbildning finde Sted hos Nogle , men den rette Selvfornægtelse kan ikke skee af egne Krcefter , uden Guds <b>Kjærlighed</b> udøses i dit <b>Hjerte</b> ved den Helligaand , men heller ikke kan Guds <b>Kjærlighed</b> komme i dit <b>Hjerte</b> , saalcrnge du elsker Verden , Joh . 2 , 15. Her kunde du sværge : naar det ene ikke kan skee uden det andet er skeet , naar jeg ikke kan elske Gud , saalcrnge jeg elsker Verden , og jeg ikke kan forncegte Verden , uden jeg elsker Gud , saa seer det ud som noget umuligt . Jeg maa svare dig , det synes umuligt for en Uerfaren og for vore egne Krcefter , derfor maae vi fFrst ville troe og lade Guds Acmd overbevise os indvortes hvad vor Forstand maa antage udvortes , at der er en Gud til , en Skaber og Opholder , hvis Almagt , Viisdom og Godhed vi ei kan begribe ; lad os beundre dette Vcrsen , fatte Lcengsel til at kunne elske ham , lad os da ikke gruble med Fornuften , men enfoldig , lydig og ved Troen opofre os i hans Villie , ikke slaae over med Fornuften , hvorledes det vilde gaae os , om vi maatte forlade den Lyst eller Begjerlighed , ' eller den Ting vi elske , nei ikke saaledes overslaae , men anbefale os ganske i Guds Villie , og bede uafladelig at vi maatte faae Naade til at troe , eiste og forlade os paa saa god og <b>kjærlig</b> en Gud , betragte hans Ord , sec hvad hans SM vor Frelser har lidt for os , bede , hans Helligaand at forklare os <b>Sandheden</b> og ikke forlade os paa vor formorkede Forstand ; naar vi saaledes lydige overgive os i Guds Villie , da stal han lade et Skimt af sin guddommelige Kjærligheds Straalc falde paa os , og naar vi faae see lidt af denne salige Skat , da er det just denne Perle , der bringer vort hele <b>Hjerte</b> til at forncegte alt for at faae kjobt eller blive Eier af denne Skat . Vel vil den Guds <b>Kjærlighed</b> Straalc tillige bringe Sorg i vort <b>Hjerte</b> over de bcgcmgne Synder , og over at vi ere urene og uskikkede til den salige Naade , men troe vi , at Jesus kom for at borttage vore Synder og rense os fra denne Nreenhed , saa stal denne Tro , efter Kamp , Tvivl og Strid , i en ret Bedrøvelse efter Gud , finde hans store

<h4>2997</h4>BFrn , Venner og Tjenere boe ogsaa deri med ham , men nnder ham , og ester hans egen Villie . Snmma : det ganstc <b>Hjerte</b> , som udkrceves til Guds <b>Kjærlighed</b> , er egentlig den Ndelelighed , Oprigtighed og Redelighed , med hvilken man begjærer Gud , opofrer sig til Gud , og Mr sin store Hovedsag af at tcekkes ham , med alle stridige Tings Forncegtelsc . Her hedder det : hvo , som elsker Fader eller <b>Moder</b> , Hustrue eller Brødre eller SMre mere end mig , han er mig ikke vcerd . Denne er da Meningen af Lovens fyrste og største Bud , om at elske Herren vor Gud af ganske <b>Hjerte</b> , af ganske Sjel og af ganske Sind . Komme vi nu til en SelvprMlse , da findes jo Nogle , som aldeles ikke elske , men hade Gud : Andre , som elske ham efter Anseelse , og af et halvt eller deelt <b>Hjerte</b> : men nogle Faa , som elske ham af ganske <b>Hjerte</b> , stjMdt endda ikke nær saa heftig , som han er vcerd . De fFrste erc hans aabenbare Fiendcr ; de Andre hans falske Venner ; det tredie Slags hans igjenfMe og sande Hjertens Venner . Joh . 5 , 42. siger Jesus nogle haarde Ord til Ilderne , nemlig : jeg kjender eder , at I have ikke Guds <b>Kjærlighed</b> i eder . Vi , - som ikke kjende hverandres Hjerter , ikke altid fcelde saadan dristig Dom . Dog findes altfor mange af det Slags Mennesker, som <b>selv</b> kjendelig nok aabenbare og blotte den samme usalige Tilstand , nemlig de , som efter Jesu Ord , Joh . 3 , 20. hade Lyset , det er Guds <b>Sandhed</b> , og altsaa Gud <b>selv</b> . Disse Vc > rkheds VMn , Guds Fieuder og Retfærdigheds Foragtere , vil jeg alene advare , at det vil blive dem tungt , ja omsider umuligt , at stampe mod Braadden . Kan Guds <b>Kjærlighed</b> ei overtale dem , da stal omsider hans Vrede fortcere dem . Siger disse Daarer nu i deres Hjerter : der er ingen Gud , Ps . 14 , 1 ; da stulle de dog i hiin Verden faa Troen i Hænderne , naar det stal hede : mig hid disse mine Fiender , som ikke vilde jeg skulde herske over dem , og hugger dem i Stykker for mit Ansigt . I , som elste eller egentlig indbilde eder at elste Gud , men hcenge ikke ved ham i Oprigtighed og af ganske <b>Hjerte</b> , I , som efter den anførte Forklaring , dele eders <b>Hjerte</b> imellem Gud og Verden , ja ere alene undertiden og udvortes med ham , men indvortes imod ham , vider , at eders Dom er den samme , ja maaste saameget vcerre , som eders Hyklerie og Lunkenhed er Gud vederstyggeligere , eud aabenbar Fiendstab ; hvorfor Aanden Mster, at I vare ret kolde eller ret varme , Aab . 3 , 15. Betcenker dette , scerdeles I , som tilforn sandelig har havt Guds

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010061506031'target=_>Funcke, Otto, 1880, O. Funcke's Husandagtsbog</a><h4>207</h4>nogen Vej i vore Dage . Hvad der maa ste , maa ste . Godt , du stal hellerikke blive forstyrret i det Altsammen .  Bind Hsjtidsoffer med Reb indtil Alterets Horn ! " Men fortab ikke din Sjcel i de ydre Ting . Tag dig dog fsrst nogle Ojeblikke og stu en Smule ind i dig <b>selv</b> og mod det Hsje . Du smykker det hele Hus , pryder luletrce og Julebord og stulde — glemme dit eget <b>Hjerte</b> ! Det vilde vcere en stem Ting . Samle dig , lceg din Travlhed tilside . Det er en Jammer , men det er <b>sandt</b> , at mange ellers elskelige <b>Kvinder</b> paa ingen af Aarets Dage ere saa uelstvcrrdige , saa heftige , saa pirrelige , og mange Kristne ikke nogen Dag saa ukristelige , som paa Kristi Fsdselsdag . Og i tusinde Huse faa Vernene fsrst om Aftenen under luletrceet et venligt Ord at hsre . Dette er dog en stem Ting ! Du maa blive Me i dit Indre . Se , den hellige , stille , sagte Susen , hvori Gud <b>selv</b> ncermer sig , vil lsfte dia . Men , hvorledes skulde du kunne hsre denne Susen , naar du <b>selv</b> larmer saa uroligt ? Nej , bed du nu Herren engang ret om , at han vil samle dia , om at han vil lede din Sjcel ind ! sin underfulde <b>Kjcerligheds</b> Midtpunkt , den <b>Kjcerlighed</b> , som idag er aabenbaret , og at han idag vil give dig mange gode Tanker , cegte himmelsk lulesolstin . la , du tan under alt Arbejde vcere fuld af Kristi Lys , vil det kun og bed kun derom . Da fsrst er Festen rigtig festlig . Og kommer du ihu En , der er ensom og ssrgende og uden Festglcede , da mcerk dig det hurtigt til imoraen , ifald du ikke ' allerede idag kan aaa derhen og bringe Julens Lys . Dette er at  binde Hsitidsoffer med Reb indtil Alterets Horn ! " Og hvis din Aand , som vil fryde sig , msder En , med hvem du er bleven uenig , og det mellem dig og ham engang har vceret varmt , men nu er koldt ,  og han har dybt forncermet mig ! " siger du , o , jeg beder dig . kom ihu , at du i den Frelser , som idag er fsdt , har faaet Tilgivelse for alle dine Synder af lutter Naade . Og mcerk dig det ikke til imorgen , forat du da kan bringe din Modstander Fred ( thi saadanne Notitser gaa fordetmeste tabt ! ) , nej , gaa straks hen og tag dit bedste Ansigt med. Saaledes  binder du Hsftidsoffer med Reb indtil Alterets Horn " , og du vil ogsaa finde , at du bagefter kommer langt rastere afsted med dit Arbejde . Og om Aftenen , naar det bliver lyst i Vcerelset , vil det da blwe endnu lysere i dit <b>Hjerte</b> . Da vil den stille , sagte Susen komme , og du vil vcere gladere end alle Bsrn , thi l dit <b>Hjerte</b> vil lyse , hvad Evangelisten Johannes strev :  Se , hvilken <b>Kjcerlighed</b> Faderen har bevist os , at vi stulle kaldes Guds Bsrn ! " -

<h4>223</h4>Og hvad den gamle Simeon netop havde profeteret om hendeK Barn ( V . 29 - 32 ) , kunde kun vcekke den stolteste , helligste Glcede i hende . Og nu dette Ord :  Ogsaa din Sjlrl stal et Svlrrd gjenncmtrange!" Hvad Skrcekkeligere kunde man sige ? Ak , altfor snart skulde hun erfare , at Simeon havde talt <b>Sandhed</b> . Fra den Dag as , da hun med sit kjone Barn i den msrke Nat maatte redde sig fra Tyrannens Svcerd ind i Hedninaelandet , indtil hun som mater Hoiorosa stod under sin elskede Fsrstefsdtes Kors , har Naadens Sol mange Gange tabt sit Skin for hende og ofte har bun maattet tcenke paa Svcerdet , hvorom Simeon talede . T > et var for Moderen Maria som en indre Dsd dette , at hun maatte give Afkald paa alle sceregne Fordringer paa sit Barn og overlade det til den yele Verden . Men ikke blot som <b>Moder</b> fslte Maria hint Svcerds Bitterhed , men ogsaa som Israelitinde . Ligesom alle hin Tids Troende Havde ogsaa hun Haabet paa en hurtig og glimrende Seiersbane for det messianske Rige . Og nu blev Kongen til et Offerlam , og nu indviede han sine Trofaste , som vilde have triumferet med ham i Herlighet ) , til Dsden . Kun Faa blev Frelseren  til Oprejsning " , Utallige . til Fald " ,  til et Tegn , som Mdc modsiges . " ( V . 34 ) . Dette blev mere og mere aabenbart . Folkene rasede imod Herrens Salvede , og vcerst rasede Israel <b>selv</b> . O , det var haarde 33 eje for Disciplene , Alle have de forarget sig derover , og kun fordi deres <b>Kjcerlighed</b> var stcerkere end deres Tro , have de iajen sundet sig tilrette . Men mangt et Svcerd gik derved gjennem l ) eres Sjcele . Meget dybsinoig har Kirken viet denne Dag til Erindringen om Stefanus , den fsrste kristne Martyr . Liae ved Siden af oen nyfsdte Konge staar det forste Blodvione , til et sikkert Tegn paa , at Kristi Riaes Udvikling kun gaar gjennem Blod og . Taarer , og at Fredsfyrften doa ikke forst er kommen forat bringe Fred , men Svcerd . Hvem der kjender Verden , hvem der kjender Mennestehjertet , hvem der i sit eget <b>Hjerte</b> daglig og bestandig sinder MotMnden mod den tuatende Naades.Oro , Han begriber oette . Og gaar det ikke Utallige , som er begyndte at fslae Kristus efter , netop som Maria ? Begyndelsen er saa liflig og pa glcederig, men saa kommer Svcerdet . — Naar du , kjcere Lceser , vilde antage dig et forceldrelsst Barn , som dit eget , ikke <b>sandt</b> , da vilde du fsrst gjsre alt Muligt forat fcenasle det til dia , forat vcekke dets Tiltro og <b>Kjcerlighed</b> , og mangen Vanart og Ulyoighed vilde du lade ustraffet ? 3 ) a derimot ) , naar dit <b>Hjerte</b> fsrst havde sundet Barnets Hzerte , da beaynder Opdragelsen . — Saaledes er det med Herren . Have vi efter tana Kjcempen , Sogen , Tvivl endelig forstaaet , hvad det vil sige , at vi have ham til Frelser , saa lader han fsrst sin hele Huldsalighed og Venkghed straale ind i Hjertet . Vi fejle imidlertid meget , naar vi tro , at det stedse vil gaa saaledes sra Glcede til Glcede , fra den ene salige Erfaring til den anden , og vi saaledes som af sig <b>selv</b> skulde vokse ind i det himmelske Liv . O

<h4>415</h4>Zn md i Hærtet , snart scenker det sig ned deri som en helbredende Himmelbalsom , eller som en Liv vcekkende Foraarssolstraale ; men overalt og altld er det den fuldkomne Visdom , som <b>moder</b> os i overalt finde vi ogsaa , hvorledes han har udforstet det mennestellcze <b>Hjertes</b> og Livs Digten og Tragten . Hvorledes skildrer han Me den crnastelige Bekymrings Voesen paa den ene Side og paa den anden Side den taabelige Lcegqen-Planer uden Gud ; hvorledes troeffer han ikke det skinhellige Hykleri og Selvbedraa med knusende Ord . 09 atter , hvor dybt skuer han ikke md i en ester Redmng skrtgende Sicel ; hvorledes forstaar han ikke under Billedet af den forlorne Ssn at skildre Millioners oa atter Millioners Letstndmhed, samt de strcekkelige Folaer af dette Letsind , og atter , hvor uforttgnelmt fremstiller han ikke t hin Prest og Levit , som gaa den halvdode Mand fork , den menneskelige Naturs Hjertelsshed ! Doa vi vllle standse , thi ethvert <b>Blik</b> i Evangelierne lader os se en Visdom hos Jesus , som aldrig var paa Jorden fsr eller efter ham . Men denne Visdom er ikke fra Himlen af scenket ned i hans SM som en fuldfcerdig Gave , nej , han har tilkjcrmpet , Medet , tilarbeidet stg den ved en stadia Ud- oa Indgaaen i Guds <b>Sandheds</b> Tempel , ved en virkelig Optukken af sin Sjcel for Guds Lyses som oplyste for ham Himmel og Jord , Tid og Evighed . Og saa v : st som Jesu Disciple skulle vcere Jordens Lys og Salt . saa vlst stulle de ogsaa blive ham lige i hans Visdom . Forfremmes du , som kalder dia Jesu Discipel , i Visdom ? Men dersom uke - hvorfor da ikke ? Prsv det <b>selv</b> — men cerligt !

<h4>552</h4>og evigt Liv " — ! Visselig <b>moder</b> man ikke saa sjelden Mennesker , der fore et saadant Sprog , men hos hvem man uvilkaarlig har den Folelse : Disse have ingen Ret til at tale saaledes , det er kun noget Tillcert . Men paa den anden Side sinder man ogsaa Saadanne , der i al Oprigtighed staa paa Tslderens StandPnkt , og som desuagtet ikke kunne komme til en ret Aandens og til Vished om deres Frelse . Just disse , som indadtil ere leste fra sig <b>selv</b> , og som gjerne vilde hengive sig helt til Kristus , som fjerne vilde glcede sig og dog ikke kunne glcede sig , fordi de vente , ieg ved ikke paa hvilken sceregM Guds Uabenbarma og Ord , som oog ikke er dem lovet , disse gjcelder , hvad her skal siges . Hvorledes kunne de da komme dertil , at de kunne glcede stg over sin Naadestand ? Apostlene svare enstemmig alle fra sig <b>selv</b> afdode Sjcele :  Ved Troen bliver man retfcerdlg for Gud " . Hvad vil da dette  Ved Troen " sige ? Nu , det vil ncermest sige : Ikke ved Gjerningerne, men videre vil det ogsaa sige , at man ikke maa vente Benaadelsen og den indre Oplysning herfra og derfra , men gribe den ud as felve Kristi Ord ; thi i Templets Marmorhaller saavelsom ved alle Hcekker og Gjcerder lader han de i <b>Sandhed</b> Bodfceroige faa nyde sin <b>Kjcerligheds</b> fulde Solglans , kalder oa lokker dem :  Saa griber doa til og tåger mig I , som ikke vente Noget afEder <b>selv</b> , men Alt af mig . Eder tilhorer al min Naade , Eder tilhsrer vet evige Liv . " Hvorofte sier det ikke , at de kjcere Mennester sige :  Ak , jeZ kan ikke brinae Guo Andet end en stakkels elendig Synder , dog Salighedens Vished fejler mig . " Men , jeg beder oig , saa bring du dog Gud den stakkels Synder oa tag frelseren fra ham ! Han staar der jo ! Netop saaledes , som du er , vil Herren have Menneskene , og dem , der ere saaledes , dem vil han alle have og fuldende Alt , Alt i dem . Saa grib dog til , dig tilhorer Jesus Knstus , oa dig til ^ horer al hans Naade . Netop dette stal du tro ; — ikke se , ikke smage , ikke fole , ikke lade Prester og andre fromme Folk forsikre det for oig , nei , du skal tro det ! Smager og foler du det ogsaa , ere ogsaa de Fromme af den Mening , at ou er et Guds Barn — desto bedre , men betragte de dig med Mistro , smager og foler du <b>selv</b> Intet , saa er dog Sagen derfor ikke anderledes . Her er Kristi Ord , scenk om Sjcel dybt deri og trods derpaa , om det saa var hele Skabningen . Og om end alle Stjerner paa din Aands Himmel aaa tilgrunde , oa om alle Belials Bcekke bruse om dig , og om Tungsind formorker dit <b>Hjerte</b> , og saamange listige Erfaringer fra dit twligere Liv aldeles fordunkles for dig , og om alle Himmelens Hellige vende dig Ryggen , saa trced du desuagtet hen for din Herre Kristus og sig : Her er dit Naadeord , og her er en arm Synder , de To hore sammen ; her er en Mand meo et sonderknust <b>Hjerte</b> , der intet Andet har at bringe end sig <b>selv</b> , men som ogsaa brmger sig helt , og der er dit Ord , o Jesus , at du vist vil lade den arme

<h4>829</h4>Disciple . Medens disse vifte fig , som de vare og som Fslge deraf maatte taale svcer Idmygelse , men tillige gjennem Herrensstraffende Ord nu ogsaa erholdt Lys over deres eget <b>Hjerte</b> og Vcesen , — havde Judas i det Sted . altid trukket sig tllbage i Msrket med sin Synd . Han bavde gjort dette med saa stor Klogskab og Fcerdighed , at hans Medoisciple ingen Anelse havde om hans Vjertes onde Bestaffenked . Enonu ved Hojtideligholdelsen af det noste Paastemaaltid, da Herren udtaler , at En as dem stal forraade ham , tcenker Enhver kgesaasnart paa sig <b>selv</b> som paa Judas . Saa lidet have de forstaaet ham . Ja , han har forstaaet saa godt at skjule sit virkelige Sindelaa ^ at <b>selv</b> Jesus aldrig kunde komme ham ncer med nogen Dadel . Men den eneste Gang , da dette blev muligt , ( nemlig i vor Fortcelling ) » havde Judas viost at drage alle de andre Disciple med ind i sit onde Tcenkescet . Desuagtet mcrrkede Judas vel , at Jesu Dadel fornemmelig gjaldt ham ; han solte , at han var ajennemstuet og afslsret af Herren . Allerede dette gjorde ham ra < sende . Dertil kom , at Jesu Ord ingen Tvivl efterlod om , at Lidelse og Dsd var en besluttet Sag ; endelig , at netop det viljelsse , hengivende Kjlrrlighedssind hos <b>Kvinden</b> var den Hjertetilstand , der ene og alene behagede Jesus og passede i hans Rige . Men Judas maatte sige sig <b>selv</b> , at hans kolde Selvsyge netop maatte vcere det stik Moosatte af denne sMel ^ e ; at han maatte vende aldeles om eller ogsaa afstjcrre det sidste Baand mellem sig <b>selv</b> oa Jesus . Han bestuttZe sig til det Sidste . Vi ville i det selgende komme tilbaae til Judas og idag kun betone , at ved Manas cegte Kriituskjcerkghed optoendes ludas ' s s ^ ristushad . Dette er endnu fremdeles saa . Hvor Herrens Disciple leve i oprigtig Kristi Efterfslgelse og bevise dette ved deres hele Tcenkemaade , Ord og Gjerning , der vil to Ting blive aabenbare : De Opriytige ( som imidlertid endnu ikke kjende Herren ) ville se saadanne Gjerninger og prise den himmelske Fader derfor . De ville netop derved anspores til alvorkgt at soge Jesus . Men de , der ikke ere af <b>Sandheden</b> , og som ville leve for deres Lyster , ville netop omvendt vcrlte Had , Vrede , Foragt . Spot og , om de kunde , — Forfslgelse over hine Jesu crgte Bekjendere . Jesus har forudsaat begge disse Ting , derfor maa det vcere <b>sandt</b> . Man mcerker vistnok i vore Dage paa de fleste Steder forbausende lidet til Forfslgelse, men hvor der er Had og Forfolgelse , der har den ofte mere sin Grund i Daarskab , Trangbrystethed og Urenhed fra deres Side , der holde Kristi Banner hsjt i det Idre , end i deres Kjoerlighedsgjerninger . Paa den anden Side gives der en Slags Ssndag.skristendom , som behager Enhver oa. ikke foraraer Nogen , fordi den : kke straffer oatugter Nogen i hans Samvittighed , — men som heller ikke opbygger Nogen , fordi den mangler Lys og Salt , <b>Kjoerlighed</b> oa Alvor . Prsv dig vel , kjcere Lceser , om maaske ikke ogsaa dm Kristendom skulde vcere saaledes bestaffen . At give Forargelse og Anstsd formedelst uren Kristendom er en strcekkelig Ting ; hvem der imidlertid ikke er til

<h4>879</h4>relse paa vort <b>Hjerte</b> , og iscer da , naar vi have nwdt den Opofrende med Kulde og Hjertelsshed , og da , naar hans Gjerning blev ham <b>selv</b> sur ! — Hvor brcender alt dette os i Hjertet ! Hvor stammer ikke da det Bnske hos os , at vi ogsaa ganske kunde hengive os til denne Mand , der med sin Person , Formue og Kraft traadte i Skranken for os ! Anvend dette paa Frelseren , ver med Legeme og Sjcel , under tusinde Taarer og Angester , Kvaler og Smerter helt har opofret sig for dig , for at fsre dig for evig , og det med Legeme og Sjcrl , ud af den dybeste Fordcervelse og Ulykke til den hveste Livsherliahed . Tror du dette ? Tror du det ikke , kjcemper ou idetmindste ikke for at kunne tro det , — saa kan du vistnok ikke trcede hen til Herrens Bord ; da er Sakramentet dig endnu et forseglet Brev . Men tror du det , saa maa det oasaa brcende i dit Hzerte :  Herre , som du hengav dia for mig , saaleoes hengiver jeg ung til dig . Ikke mere lever jeg for Synden , om jeg end saa ofte ligger under for mit gamle Onde , men for dig ; ikke mer for mig lever jeg , ikke mer for Verden , men for dig , der er dod og opstanden for mig . Ja , du stal vcere mit Livs Herre . Men Langmodighed maa du have , megen Langmodighed ; Kraft og Naade maa du give , megen Naade , thi jeg er svag og fattig . Langsom var min Fremgang i Helligmorelsen , ja , ofte syntes det mig , som om Alt g : k tilbgge . Vanskeugt bliver det for mig at lade dine Veje vcere velbehagelige for mine Ojne , og ganske uovervindelig er min Gru for Lidelsen . Men jeg vll dog vcere om alene , du min Frelser! Og fordi jea vil vcere din , derfor vil jea ogsaa , som du har mort , leve for Menneskene , som ere kaldte med nng til den samme Herlighet ) . Alt Had , Nid og Uforsonlighed vil jm fordsmme og saa <b>Kjcerlighed</b> og bringe Fred , hvor jeg kan ; men Naade og Kraft oertil maa du give mig , og ogsaa derfor kommer jeg til dit Boro , som din Haand har dcekket . " Ikke <b>sandt</b> , det er en simpel , indlysende Sag ? Du stal kun elskende lcegge dig op til Frelseren , ligesom Barnet tll sin <b>Moder</b> ; det er alt. Hvor der er oprigtig Kjcerlighedstrang , der er ogsaa Nlrrdighed ; th : <b>Kjcerligheden</b> swder ud Alt , som hindrer Kjcerliaheden. Hvor der er ren <b>Kjcerlighed</b> , der er ogsaa Frimodigyed , thi den fuldkomne Kjcerligheo drwer Frygten ud . Saa afklced din Sjcel al Egenretfcerdighed og Usandhed , og il saa uden Tvivl og Nslen til Herrens Bord ; thi for dig er Herrens Bord dcekket .

<h4>1235</h4>give . Ren <b>Kjcerlighed</b> f . Eks . hos trofast elskende Forlovede eller Wgtesolk er uden Tvivl en kostelig Ting , der er noget Forcedlende og Lutrende deri , og mangen ond Drift og Dragelse af den lavere Natur holdes derved borte . Men det er latterligt eller endog bespotteligt, naar man , lad det end vcere i de mest henrivende Ord , synger oa taler om den jordiske <b>Kjcerlighed</b> , som om den kunde tilfredsstille Menneskehjertets inderste og hemmeligste Trang . Det er kun Selvbedrag og Forblindelse , — den nsgterne <b>Sandbed</b> kommer ofte frygtelig snart bagefter . Hjertets fuldkomne Hengivelse kan kun finde Sted der , hvor det msdes af en fuldkommen Optagen , og hvor al lHjertetrang fuldkommen forstaaes og fuldkommen tilfredsstilles. Men jo dybere og oriyinalere to Mennesker ere anlagte , desto tidligere og desto oftere vil der ogsaa i deres Mgteskab komme Tider , hvori den Ene ikke ganske , forstaar den Anden , ikke kan fole med ham , ikke folge med ham , lkke lide med ham . Det maatte Abraham erfare hos Sara , David og lob hos sine Hustruer , Jesus hos sin <b>Moder</b> . Ja . ikke blot det , at vi ikke formåa ganske at forståa hinanden , men der gives ogsaa hos ethvert Menneske Gebeter , hvor han ikke gjerne vil forstaaes , hvor han skjuler sig med Hensigt , endog for sin fortrolige Medhjcelv . Ingen Mgtemand , ingen Wgtehustru vilde ( naar de svorge sin Samvittighet » grundig tilraads ) onste at vcere absolut gzennemsisstige for hinaudeu , ganste og aldeles kjendte og ganske oa aldeles til at kjende . - Fremdeles gwcs der . ogsaa i de lykkeligste Mgtefkaber , Dissonanser og Sammenstsd , thi der er to Viljer , som , i bedste Fald , ofte drives af Selvsyge og alskens urene Bevceggrunde Og iscer , hvor Meget mangler der ikke i , at <b>selv</b> den mest trofaste jordiske Kicerliahed kan stille vor Trang og udfylde vore Lcrngsler , — dette vort <b>Hjertes</b> Skrig eftcr fuldkommen Erkjendelse , ester Frihed fra Synden , efter evigt Liv og uforgjcenaelig Herlighed — ! I vor dybeste Nod . i vor dybeste Lcengsel staar dog <b>selv</b> den mest trofaste jordiske <b>Kjcrrlighed</b> fattig og afmcegtig ved vor Side , og naar oen da grceder og lider meo os , gjor den Alt , hvad vi kunne vente af den . Jesus alene , > han , i hvem Guddommens Fylde bor , og i hvem Menneskehcdens hsjestc Ideal er opnaaet , — han . som er hjemme i vort Vcesens dybeste Dyb og paa det Fuldkomneste foler med os . — han , som ganske og aldeles gjennemskuer os og dog ikke forkaster os , men tun elsker saa meget mere medlidende og taalmodig , — han , som har gjort al vor Nsd til sin Nsd og vil fsre al vor Lcengten til dens Maal langt over , hvad vi bede og forståa , — han alene kan krceve fuldkommen udelt Hjertehengivelse . Og i Virkeligheden , han forlanger den af dem , som virkelig kjende ham . Intet Mindre end denne absolute <b>Kjcerlighed</b> mener han , naar han sverger Petrus , dig og mig :  Elster du mig ? " Hvorledes lyder dit Svar ?

<h4>1805</h4>Saaledes havde endnu Ingen spurgt denne Mand , og det syntes ogsaa at vcere et overflodtgt Sporgsmaal ; thi Alle , som kom til Bethesda , vilde blive helbredede , og sterlig denne , der allerede havde voeret syg i otte og treti Aar , og ' dog ikke var vegen fra Stedet . Men just fordi Ingen havde spurgt ham saaledes , sverger Jesus ham . Han drager derved den Syges <b>Blik</b> hen paa den underlige Sporger , der ' beviser ham en saadan hjertelig personlig Velvilje . Det er dette , vi overalt stulle se og lcere hos Jesus . Han bringer aldrig blot en Gave eller Hjcelp , men altid sig <b>selv</b> , Hjcetperen , Gweren , sin <b>Kjcerlighed</b> , sit <b>Hjerte</b> . Han vil give Enhver, som han qiver , tilligemed Gaven sig <b>selv</b> . Dette skulde ogsaa vi altid betcenke , hvor vi ville berede bedrovede Mennesker Hjcelp og Vederkvegelse . Ak , hvor mange af dem , som give og hjcelpe , saare ikke ved sin  Hjcelp " mere , end de . gjere godt . De give saa fornemt , saa nedladende , — forske saa politimcessig efter gamle Synder som Lidelsens Aarsag , Noget , den hellige Jesus aldrig giorde , — mene , at de paa Grund as sin Gave tor blande sig ind i alle dens Forholde , som de hjalp . — Eller , naar det ikke gaaes frem paa denne Maade , saa bringe vi dog den lidende Medbroder saa sjelden os <b>selv</b> , vor fulde Kjcerltghed og personlige Deltagelse . Men denne Sjcelens fulde , medlidende Hengivelse , forst den kan virke Noaet , fsrst den kan smelte det stove <b>Hjerte</b> og lukke det op for Guds Naade herovenfra . Derved blive ogsaa smaa Gaver store . Thi til en <b>sand</b> Velgjerning horer ikke saa meget Penge som megen <b>Kjcerlighed</b> . Denne <b>Kjcerlighed</b> gjsr ogsaa opftndsom , og det Olle bliver oa meget , fordi det kommer til rette Tid , paa rette Sted , paa den rette Maade og i den rette <b>Kjcerlighedens</b> Aand . Dette Sidste er Hovedsagen , og derved vinder det Jordiske et evigt Vcerd , fordi det kan virke det Evige . Derfor blive Jordens Goder og Mdelser forst da i <b>Sandhed</b> velsignede og nyttige , naar vi i dem erkjende og smage  de gode Gaver " og Velsignelser fra Faderen i det Hoje . Saaledes maa ogsaa de Mennesker , som du vil vcere til Nytte , i hvad du gjsr for dem , smage Noget af den himmelske <b>Kjcerlighed</b> ; forst da vkver din Hjcelp til Velsignelse , og da vil den drage hjerterne opad mod Himmelen . Tcrnk paa at hjcelpe saaledes , og oa vil du blive glad derved paa Jorden og i Himmelen .

<h4>1928</h4>Sundhed , og den er jo dog blot et forgjcenaeligt Gode . Nei , tik bern blot , som gjennem Kjærlighet ) og Lidelse , gjennem Fald og Oprejsnma , gjennem Synd og Bod ere traadte i en personlig Livsforbindelse med ham , Redderen , som den « store Synderinde " og den samaritanste Spedalske og som denne <b>Kvinde</b> , — til dem kun siger han :  I ere frelste " ; tyi den almcegtige Hjcelper var bleven oeres Ven og Broder . Denne Visyed , at vi med Liv oa Sjcrl , i Liv og i Dsd ere Jesu Ejendom , denne Vished , at i Tid og Evighed Intet , <b>selv</b> ikke alle Livets Storme , stal kunne Me os fra hans Kjcerliahed , — denne Vished , at vi , trods alle vore Svagheder og Skrsbeligheder , Synd og Vanart , dog formedelst ham endelig og tilstut stulle komme til Guds Boms fulde , salige Frihed og Herlighet ) , — denne indre urokkelige Vished er det , som Herren kalder Fred . Denne Vished turde <b>Kvinden</b> have , da hun kncelede ned foran den velsignende Jesus , og derfor sagde ogsaa Herren til hende :  Gat bort i Fred ! " Gjennem Sygdom og Helbredelse havde hun lcert at ssge og finde sin Sjcels Frelser ; lysende stod fra nu as hans Billede for hendes Sjcel , og han fsrte hende ogsaa kjcrrlig gjennem den morke Dsdsnat ind i det himmelske Hjem . Og du , kjcere Lceser , hvorledes har du sundet Herren ? Gjennem Glcede eller Lidelse , paa jevne , bwde Veje eller i den varme Strid ? Hvorledes har du sundet ham ? Eller har du overhovedet endnu ikke fundet ham ? Maaste er hans Skikkelse dig enonu fjern og fremmed , indhvllet i Taage , eller den ser mere truende end velsignende paa oig . Da sorsi vel efter , hvad Grunden er , og om oit <b>Hjerte</b> ogsaa ret ssger efter Guds <b>Hjerte</b> . Ikke lcenge vil du ssge forgjceves , hvis du , som <b>Kvinden</b> , kommer bsnfaldende ; ikke loenge vil du blive uden hans omskabende <b>Kjcerlighed</b> , hvis du beder med Digteren : Kom du Folte-Frelser <b>sand</b> , Fsdt of Ms foruden Mand ! Med din Guddoms Valde lom .

<h4>2367</h4>Sygdomme og have sjelden etler aldrig Fslelsen af den freidige Ungdom . Men ogsaa hos de mest Begunstigede har det en Ende med Kraft og Sundhed ; ligesaa aftager ogsaa Meisens Varme og Fantasiens Kraft med Aarene . Dog kan ethvert opngtig Gudsbarn faa erfare , at af Samfundet med Ond , som er den evige Ungdom , tilflyder der ogsaa ham en stadig , indre Foryngelse , saaledes , at han . stjsnt han celdes , dog forbliver indvortes una , dygtig og kraftig . ( Psalme 92 , 14 — 16 ) . Den , som lever i Gud , han lever ogsaa i de stsrste Tanker , som omsftcende Himmel og Jord , Tid og Mighed . Den , som lever iog af Gud , han lever ogsaa i det saligste Haab , der gjor Nutid og Fremtid lys og klar . har du ikke erfaret , hvorledes den ensommeste og tsrreste Orken blev til et rigt , smilende Paradis for dig , naar du havde dit Livs Centrum i alle Tings Ophav og Middelpunkt ? Erholdt du ikke i Tider , da du trykkedes af tunge Byrder , en underfuld Spcendkraft og Friskhed , saaat du hcevcde dig paa Vinger som Vrnens , naar du i barnlig Bsn havde soenket dm Sjcel ned i al Livets Kilde ? O , salic ; den , som har dette Bevis for Kristendommens <b>Sandhed</b> i sit <b>Hjerte</b> z ham kan ingen Vantro rove hans Krone ! Men den , som lever : Gud , alle Aanders Fader , han vil ogsaa just fra ham modtage Lys og Kraft , saaat han ( ligesom Paulus ) overfor sin ? Medmennesker kan fastholde <b>Kjcerligheden</b> og Haabet . Vi have jo ogsaa alle mer eller mindre at gjsre med , som endnu staa ganske fjernt fra Kristendommen, ja , som vel endog angribe og bekjcempe vor Tro . I Selskaber , i din Kaldsgjernmg , paa Reiser , ved Bryllupper , ved Begravelser og ved mange andre Anledninger maa du vcere forberedt paa at maatte gjore Regnstab for din Tro . Og du skulde saa gjerne gjsre Mere end dette , — du skulde ogsaa gjsre Tvivleren til en Troende . Men ofte horer man denne Klage :  Ak , vi kunne aldrig finde de rette Ord og den rette Maade ! " la , visselig , ligesaa vigtia som denne Sag er , ligesaa vanskelig er den ogsaa . Almindelige Regler lade sig ikke her yive . At vort stcerkeste Vidnesbyrd maa beståa i vor Vandel , i vort hele Sindelag , i vor <b>Kjcerlighed</b> , i hvad vi ajsre og ikke gjsre , det forstaar sig af sig <b>selv</b> . At disputere er ttke Hvermands Sag . <b>Kvinderne</b> er det sjelden givet , og sjelden anstaar det dem ogsaa . Ogsaa mange Mcrnd gjsre ofte bedre iat sige : « Jeg kan ikke tale Meget derom , men jeg ved , hvad der gjor mig lykke Ug . " Navnlig maa de , som let blive heftige , vaage over sig <b>selv</b> , ellers gjsre oe mere Skade ? nd ' Gavn . Kun Faa er det givet at tale alvorlig og indtrcengende uden at saare . Dette stulle vi lcere af Paulus , og vi ville snarest lcere det , naar vi som han have en fast Tillid og godt tzaab om vore Medmennesker , naar vi , saalcenge det er mueugt , tcrnke det Bedste om dem og kun drives af den <b>Kjcerlighed</b> , som eftertragter Alles Frelse . Den , som ved , hvorledes han atter og atter maa af Kristi Taalmodighed , Langmodighed og <b>Kjcerlighed</b> , for ham skulde Me dette vcere vcmsieligt !

<h4>2455</h4>Den forlorne Son havde bragt den sidste Rest af sin Familjearv til Pantelaaneren ; Alt var nu opbrugt .  Han degyndte at llde Mangel . " Dette var Guds anden Banken paa hans <b>Hjertes</b> Dsr . Hans Klcedning er laset ; hans hele Vcesen er blevet ustadigt og flyatende ; hans sty <b>Blik</b> forraader hans onde Samvittighed ; i hans Ansiat lcese vi kun disse to Ord : Hunger og Fortvivlelse . Aldeles forlaot staar han der . Forladt ? Hvor vare oa hans Venner , som for havde levet hojt paa hans Bekostning , som saa ofte havde drukket en Skaal for ham og med stor Begejstring svoret ham  Troskab indtil Dyden ? " Hvorfor hjcelpe de ham ikke nu ud af Nsden ? Ja , hvorfor ikke ? Spsrg heller , kvorfor de stulde have gjort det . Hvor skulde vel de , som kun levede for sig <b>selv</b> , have lcert den opofrende , sig selvforglemmende <b>Kjcerlighed</b> at kjende ? Hvorledes skulde de , hvis hele Liv var en stor Lsan , fole sig bundne ved sine Smigrerier oa Fraser ? Naar Vellysmngerne have udnyttet hverandre , lade oe hverandre lobe . Hensiaten er jo da opnaaet. Hvad de kalde Kjcerliahed og Trostab , holder kun saalcenge , som den Ene kan profitere af den Anden . <b>Sand</b> KjoerKghed og <b>sand</b> Trostab kunne kun findes der , hvor den gjensidige indre Agtelse er , idet man betragter Mennesket som et til evigt , helligt , guddommelig! Liv kaldet Vcesen . Men hvor man ikke kjender en Mennestesjcels evige og uudsigelige Vcerd , og ikke kjender Guds Frelses- og Naadeshensigter med alle Mennesker , der er ingen paalidelig Grundvold sor <b>Kjcerlighed</b> , Troskab og Samfund . — Det Menneske , som kun lever for denne Verden , kan ikke fsle <b>sand</b> <b>Kjcerlighed</b> . Han maa ester Sagens Natur vcere en fuldstomdig Egoist . Han lader , dersom det ikke stader hans egen VEre , sin Forretningsven , sin Kollega , sin Hjertensven rolig omkomme i

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111400039'target=_>Nissen, Johan, 1863, J. Nissens Samtaler over Den bibelske Historie</a><h4>439</h4>vcere uden Skremt . Hvad vil det nu sige ? Anviisning til virksom <b>Kjcerlighed</b> : laan i Skolen ; hjcelp , hvor Du stal have Opsyn og see efter ; er en fattig Nabo syg , saa bed din <b>Moder</b> om Mad til ham ; gaa gjerne et ZErende for Andre ; hjcelp Tjenestefolkene , uden at det er Dig befalet . Saaledes fremkommer det : at give de Hungrige at spise , at klcede de Nygne o . s . v. Gaa hen og gjor dette , siger Jesus til den Skriftkloge , som vilde han sige : Tal ikke meget , men hjcelp . 2 ) Mangen En vilde vel i Abrahams Sted ogsaa have ydet virksom Hjcelp , men havde havt andre Tanker i sit <b>Hjerte</b> : Kong Kedorlaomer og de tre Konger med ham have flere Folk end jeg , hvad der vel havde sin Rigtighed ; jeg udscetter mine Hyrder og Hjorde for Fare , jeg kunde <b>selv</b> blive fangen og drcebt . Alt det tcenkte Abraham ikke . Fyrst saae han hen til Nyden . Derncest til Gud : Det er noget Godt , jeg har for , Gud vil hjcelpe og staae mig bi . Det var Tro Saaledes var hans <b>Kjcerlighed</b> fuld af Tro : Gud vil staae mig bi ; hvad jeg ikke kan , vil han gjere . Men nu kjendte Abraham sin Gud , V . 22 , som den hsieste Gud , der besidder Himmel og Jord . Havde han den inderlige Bevidsthed : Gud den Hyieste er paa min Side , saa havde han ingen Frygt mere . Troen giver Mod , gjer modig . Saaledes har det altid vceret : hvor der var <b>sand</b> Mennestekjcerlighed , der fremgik den af Troen . Hvad vil det sige ? og var ledsaget af Mod . Apostlene vilde bringe Alle Christendommens Velsignelse , de havde Menneskekjcerlighed . Havde de feet hen til deres ringe Person , vare de vel blevne hjemme ; den Tro fyiedes dertil : Gud vil hjcelpe , Christus er med os . Nu fylte de sig stcerke , modige , og ligesom trodsenve siger Paulus : Hvo vil stille os fra Guds <b>Kjcerlighed</b> o . s . v. , og han har alle Ting , som besidde Magt , under sig . Luther , som af <b>Kjcerlighed</b> til Menneskene udfyrte mange svcere Arbeider ( ncevner Noget ! ) , sagde , da han havde en svcer , betcenkelig Sag for : Sagen staaer i Hans Haand , der med frit Mod tyr sige : Ingen stal rive dem ud af mine Hcender . Jeg vilde heller ikke ynske , at den stod i vor Haand . Jeg har havt mange Sager under Hcender og har tabt dem alle , men dem , jeg har kunnet kaste paa Gud , dem har jeg endnu aldrig tabt . — Saaledes endnu . Hvad man begynder i Guds Navn , det har ogsaa Fremgang i Guds Navn . Derfor , vilde Du gjerne elske andre Mennesker og gjyre Noget for dem , som Abraham , Paulus og Luther , saa plant fyrst Troen i dit <b>Hjerte</b> . Hvorfor ? Det giver Mod , Tillidsfuldhed , hvor der er noget Svcert at gjyre . Ps . 18 , 30 : Ved Dig kan jeg lybe gjennem en Krigstrop , og ved min Gud kan jeg springe over en Muur . 3 ) Men Abraham kunde maastee tcenke : Jeg vil vove det , Fjenden har meget Gods , jeg vil o ^ erfalde ham og tåge det fra ham , forat jeg kan blive endnu rigere ved hans Skatte ? Nei . Thi da Kongen af Sodoma fagde : Giv mig Folket og tag Godset til Dig : da sagde Abraham de smukke Ord V . 22. Altsaa han sygte i den hele Sag ikke sin egen Nytte ( egen-nyttig , u - ) , Abrahams <b>Kjcerlighed</b> var uegennyttig. Det er <b>sand</b> <b>Kjcerlighed</b> . <b>Kjcerligheden</b> syger ikke sit Eget . 1 Cor . 13. — O , hvor ofte sinder man nu det Modsatte ! En smukt klcedt Mand beder et Barn om at vise ham Vci etstedshen , og en Haandvcerkssvend beder om det Samme ; en Fragtmand tåger Den

<h4>462</h4>var kommen langt bort fra Gud , men , siger Augustin <b>selv</b> senere , min <b>Moder</b> grced mere for mig , end Medre gramde over deres Berns Lig . Da sagde en Biskop til hende , at hun blot skulde blive ved at bede , en Ssin af saamauge Taarer kunde umulig fortabes . Hun gjorde det stadigen — og hendes Syn blev frelst . ( Augustins Bekjendelser , som ingen Lcerer skulde lade vcere at lcese . Et Udtog deraf iI . G . Muller , Selstbekeuntnisse merkwurdiger Manner , B . 2 ) . Hvilken Kraft til Benherelse Luther tilskriver Forbpnnen , see vi af hans Breve , der ncesten alle slutte med et : Beder for mig . Vor fromme Scriver siger : Min fromme <b>Moder</b> har jeg ofte hort bede i min Ungdom , iscer tidlig om Morgenen . Hun havde bestemt mig allerede i Vuggen til Guds og Kirkens Tjeneste . Naar hun bad for sine Bern , det ene ester det andet , og kom til mig , bad hun meget hjertelig og ivrig om , at Gud vilde lyksaliggjere og « druste mig med Viisdommens og Herrens Frygts Aand o . s . v. " Hvorledes denne Ben er hyrt , derom kunne de fortcelle , som nogensinde have lcest Noget af denne aandrige Maud . — ( See Schatzkammer der christlichen Kirche . Dresden 1835 , B . 1 og 2 ) . Saaledes skulde nu ogsaa Skolen bidrage Sit til at fremkalde en hjertelig Forven hos Vernene . Lcereren maa <b>selv</b> ved passende Leiligheder gjere hjertelige Forvenner , saaledes som de ere angivne i efterstaaende Samtale . Saadanne Venner , som Lcereren maa holde udeu Kunst , af frit <b>Hjerte</b> og med <b>sand</b> Andagt , vcekke ogsaa Vernene til at bede med. De lcere tillige ogsaa , hvorledes man skal agte paa Livets Forhold , og hvorledes den christelige <b>Kjcerlighed</b> faaledes antager sig fremmed Ned , som var den Ens egen . Da lceres det ikke blot , hvad der heller ikke tyr mangle , men ved saadanne Forvenner lever man sig md deri , og Kirkcbennens saude Betydning bliver saameqet snarere opfattet og forstaaet .

<h4>466</h4>Personer , for hvem vi stulle bede . Abraham beder her for de Ulykkelige. Gjor Du ogsaa saaledes . Iblandt dine Naboer , dine Slcegtninge, der , hvor Du boer , er der En , som er haardt syg ; bed for ham ; bed for de Fattige i den kolde Vinter , at Gud maa give de Rige et velgjørende <b>Hjerte</b> ; mener Du det cerligt , vil Du ogsaa <b>selv</b> hjcelpe , som Abraham hjalp , hvor der var Nod ; siig , naar det regner meget i Hostens Tid , eller under anden Landeplage : Vefljerm os , som Du ene kan , velsign paany vort Folk og Land ; o , trost os altid med dit Ord , afvend den Landeplage stor ; giv os tilsidst en salig Dod og hisset hos Dig Glceden sod ( Psalme 115 , V . 4 ) . I en stor Torke ; for Kongen , iscer paa hans Fødselsdag ; for al Bvrighed , for alle Mennesker ved Nytaar ; at Christendommen altid maa udbrede sig videre og Gudsfrygten stedse tiltage iblandt os , naar Du beder den anden Bon , seer en Bibel liggende , eller hvor Du ellers mindes derom ; Du stal i din Morgen- og Aftenbpn ikke blot bede for Dig <b>selv</b> , men ogsaa for dine Forceldre . Hvorledes vilde Du vel sige om Aftenen ? Din Fader har lidet Arbeide , hvorledes vilde Du bede ? Din <b>Moder</b> stal reise bort ; dine Forceldre ere sorgmodige over Et eller Andet , og Mennefletrost forstaaer ikke ; en Broder har faaet en stem Vane , en Syster er paa stette Veie ; hvor saaledes Sjcelen er paa stette Veie og ligger i Synden , der stal der iscer fkee Forbon , som den Herre Jesus gjorde paa Korset ; Forbonnen i Kirken : Vi ville i vor Bon indeslutte den Syge der og der . Hvad stal det til ? Gjor det da , bed med. Nu have vi altsaa opregnet en Mcengde Tilfcelde , hvor der kan stee Forbon . Ncevner dem . Naar det da er Eders Alvor med Forbonnen , ville I let <b>selv</b> kunne sinde saadanne Tilfcelde og mange flere andre . 1 ) Forbonnen for Andre lader sig nu let befale , men det gavner Intet , naar Hjertet ikke forud er beredt dertil . Vor Historie viser os nu , hvorledes man gjor sit <b>Hjerte</b> tilboieligt til Forbon . Vi have allerede tidligere feet , at denne Sodomas Ulykke gik Abraham til <b>Hjerte</b> . Menneskekjcerlighed , det var ogsaa Kilden her , hvoraf hans Forbon fremgik . Havde Byernes Undergang vceret ham ligegyldig , saa havde han aldrig aabnet Lceberne til Forbon for dem . Christendommen forbereder nu ganske i Scerdeleshed Hjerterne til Forbonnen . Jesus har elsket mig saa hoit . Jeg vilde gjerne elske ham igjen . Hvorledes stal jeg gjore det ? Jesus henviser mig til Menneskene : Hvad I have gjort mod Een af disse mine mindste Brodre o , s . v : Match . 25 , 40. Jeg hjcelper , tjener , giver , mere kan jeg ikke gjore . Jo , den , som gjor Forbon , gjor endnu mere . Han tåger den hoieste <b>Kjcerlighed</b> til Hjcelp . Saaledes maa Du altsaa see til , at Du i <b>Sandhed</b> elsker Andre , saa vil Forbonnen folge af sig <b>selv</b> . Hvo , som beder for Andre , hos ham fee vi ogsaa , hvad han tcenker om sig <b>selv</b> . Abraham var en af de Rigeste i Landet , der ofte havde med Konger at gjore . Her beder han , han indseer altsaa , at han ikke kan give det , han kalder sig , V . 27 : Stov og Aske . Stov og Aske er noget Ringe , som agtes for Intet . Ved denne Forbon kaste vi altsaa et <b>Blik</b> ind i Abrahams <b>Hjerte</b> , hvorledes han dommer om sig <b>selv</b> . Han anseer sig <b>selv</b> , hvad han har og er , for noget Ringe , er ydmyg . Uden Ddmyghed er overhovedet ingen <b>sand</b> Bon og heller ingen Forbon

<h4>1127</h4>til . Foler , da David ikke Noget efter Gjerningen ? loab sender ctßnd , som forteller om Krigen , og som i V . 24 ftier til : Dertil er Urias , din Tjener , ogsaa dyd . Da maatte hans Samvittighed dog sige ham : Det har jeg gjort . Men han lader dog , som han ikke havde hmt det , og taler , som det ikke videre angik ham . I V . 25 siger han til Bndet : Saa stal Du sige til Ioab : Lad ikke den Gjerning synes ond for dine Mne , thi Svcerdct fortcerer paa den og paa den Maade . Derpaa tåger han Bathseba i sit Huus , og hun bliver hans <b>Hustru</b> . Saaledes hengaaer der Aar , uden at han spger Gud .  Din Synd den gjsr Dig blind og lam , fortjener Guds og Verdens Skam og brmger Helvedpine ; den smager spd udi div Mund , bli ' er bitter dog om foie Stund , saa lagrer Herren Sine . " ( Ps . Nr . 404 , V . 1. ) — Det stulle vi nu strar faae at see . 2 ) Guds Forhold . David vilde altsaa ikke ud as sin Synd , i hvilken han nu havde gaaet allerede i Aar . Han siger <b>selv</b> , Ps . 32 , 3 : Der jeg vilde tie , fortceredes mine Veen ; thi din Haand blev Dag og Nat svar paa mig . Altsaa den onde Samvittighed : Jeg har overtraadt Guds Gud , Gud veed det , jeg er forkastet af Gud . Saaledes er det altid gaaet Syndere . Cain . Hvorledes ? Es . 57 , 21 : Der er ikke Fred , sagde mill Gud , for de Ugudelige . Hvorledes forholder Gud ' sig nu ? David bekymrer sig ikke om Gud ; Gud skulde saa heller ikke bekymre sig om ham . ' Nei . Saasandt jeg lever , siger Gud , jeg ingen Synder swder ud — Ps . Nr . 416 , V . 1. Bor Historie viser <b>Sandheden</b> af dette Vers . Gud gjpr det fyrste Skridt . Han gaacr efter dem , som gaae bort fra ham . Gud sender Nathan . Han fortceller en Historie . Hvilken ? David aner endnu ikke Noget . Da hedder det reent ud : Du er Manden . Og nu taler han videre til hans <b>Hjerte</b> . V . 7 : Jeg salvede Dig til Konge og udfriede Dig af Sauls Haand og vilde have lagt Saadant og Saadant til . En Erindring om alle Guds Velgjeruinger , hvorfor Gud vel havde kunnet forlange Tak , <b>Kjcerlighed</b> og Lydighed , og dog havde han holdt til med Synden . V . 9 : Hvorfor har Du foragtet Herrens Ord ( Bud ) , at Du gjorde det , som er ondt for hans Aine ? Altsaa uden at mindes Guds hellige Ncervcrrelse . V . 14 : Du har kommet Herrens Fjender til Hjcelp til at bespotte ved denne Gjerning ; altsaa ogsaa derved forfort Andre . Derpaa fplger Forkyndelsen af Ulykken : Jeg vil opreise Ulykke over Dig af dit Huus . — Saaledes indeholde disse Ord en Paamindelse om Guds <b>Kjcerlighed</b> , Attestedsncervcerelse og straffende Retfcerdighed , som er bleven foragtet ved Synden . Hvorfor gjpr Gud dette ? Hvad stal det til ? ' Rom . 2 , 4 siger os det : Veed Du ikke , at Guds Godhed leder Dig til Omvendelse ? Den stal bevirke Erkjendelse af Elendigheden , og at man omvender sig fra Synden , erkjender og bekjender den . — Gud gaaer saaledes endnu daglig efter Synderen . Hvorledes ? Han lader ham leve , giver ham meget andet Godt , fender ham en Paamindelse ved en Prcediken , en Ven , en Lidelse . Hvad stal det til ? <b>Kjcerlighed</b> frembringer allerede hos Mennesker Gjenkjcerlighed . Saaledes der . Og Dette eller Hiint , som Gud endnu ovenikjpbet har stjcenket os Christne ( hvilket ? ) , stal bringe Mennesket til at udraabe ,

<h4>1195</h4>1 ) Goderne . Salomons Gang og Offer vare hsitidelige , 2 Kryn . 1 , I — 6 , ligesom hans Faders Gang , da han gik for at afhente Pagtens Ark . Dette behagede Gud , og han tillod ham at udbede sig Noget as ham , et Gode . Salomcn beder om et forstandigt <b>Hjerte</b> til at stille mellem Godt og Ondt , 1 Kong . 3 , 9 , altsaa om Noget , der har Hensyn til Guds Aand . Guds Aand <b>selv</b> har Forstand til at stille mellem Godt og Ondt , eller udtrykt paa en forstaaeligere Maade : den er hellig . Hellighed er <b>Kjcerlighed</b> til det Gode og Afsty for det Onde . Salomon udbeder sig Noget af Guds Aanr , han beder om et aandeligt Gode . Aandelig , hvad der er Aanden ( Guds Aand ) lig , deri findes og deraf meddeles til vorAand . Gud er <b>Kjcerlighed</b> . Guds Aands V < esen er altsaa <b>Kjcerlighed</b> , fprst til sig <b>selv</b> som den Allerhoieste . Rom . 5 , 5 : Guds <b>Kjcerlighed</b> er udpst i vore Hjerter ved den Helligaaud, som er givet os . Hvad vil det sige ? <b>Kjcerlighed</b> til Gud er det vigtigste aandelige Gode , deraf folge de andre aandelige Goder , som ere ncevnte i Gal . 5 , 22. Hvorfor kaldes de der  Aandens Frugt " ? Hvorfor er nu Idmyghed et aandeligt Gode ? 1 Kong . 4 , 29 — 34. Salomon vidste altsaa Meget , havde megen Forstand . Det viste sig ogsaa i Striden mellem de to <b>Kvinder</b> . Hvorledes? Han vidste at udftnde <b>Sandheden</b> , hvad mangen En ikke vilde have formaaet . Den , som veed Meget , har ogsaa et Gode . Naar man veed det , som man veed , ifplge Grunde og i en vis Orden , er det Videnskab . Nogle Videnstaber : Naturvidenstab . Hvad er det ? Retsvidenskab. Mathematik . — Fremdeles , Salomon lod Meget bygge , som vi snart stulle faae at hore , og Bibelen rofer det , han lod bygge ,

<h4>1231</h4>* ) Hvad Salomon erfarede , have mange Andre erfaret , iscrr Konger , som vare i Besiddelse af Verdens Goder . Den franske Konge Ludvig den fjortendes Liv frembyder mange Sammenligningspunkter . Man see i Beckers Ver- denshist . 9 Dccl : Ludvigs sidste Dage . Fru Maintenon maatte altid tcrnke paa nye Adspredelser , for blot at pirre Sandserne hos den gamle Konge , som havde nydt Alt , og dog vilde det Altsammen ikke forstaae . Hvor <b>sandt</b> skildrer ikke Lord Byro n ( Hebr . Gesange , Th erem in Abendstunden , B . 2 ) Salomons <b>Hjerte</b> , da det var i Besiddelse af Verdens fulde Nydelse , saa- ledes i en Samtale med fig <b>selv</b> :  3 Erens Krone var engang min , og Viisdom og Magt over Fjcnder . Af gyldne Bcrgre dråk jeg Vim , omgiven af deilige <b>Kvinder</b> ; mit <b>Hjerte</b> jeg baded ' i deres <b>Blik</b> og felte af Elskov det glede . Hvert Hjertets Bnske jeg opfyldt fik , og Lykken kom mig imede . — Jeg lader den henrundne Tid forbi Sjcelens Vie drage . Mon jeg da atter ensker hid de hellige Glcedcns Dage ? Ak ! ingen Time , ei nogen Dag fandt Hjertet Fred og Hvile ; det trykkede mig med lenligt Nag , det Guld , som i Kronen mon smile . — Slangen , der kryber i Skoven gren , over den kan Manden byde , men den , som stikker Hjertet i Len , hvo kan dens Braad afbryde ? Den lytter ei til Viisdommens Ord , ei Strengeleg kan den for- jage , den stikker i Hjertet , hvor den boer , og Sjcelen maa det fordrage . " — Religiese Mennesker derimod have aldrig felt nogen Tomhed eller Utilfreds- stillelse i deres Glcrde over Gud , i deres <b>Kjcrrlighed</b> til Gnd . Hvor <b>sandt</b> er dog det , som Christus siger om det levende Vand , hvorefter vi ikke mere terste , som fremvcelder til det evige Liv !

<h4>1977</h4>Paa samme Maade forholder det sig med Christendommen . Sverge vi , hvorledes er det menneskelige <b>Hjerte</b> beslaffent af Naturen , d . e . uden Christendom , saa staaer Svaret i Eph . 2 , 3 : Iblandt hvilke ogsaa vi Alle — . Altsaa af Naturen Vredens ( Guds Vredes ) Bsrn , gjere Kjpdets , d . e . det syndige <b>Hjertes</b> Villie , vandre i vort Kjods Begjcerligheder . Hvorledes bliver nu det syndige <b>Hjerte</b> bestaffent , naar Christendommens Snurdeig kommer md i det ? Det staaer i V . 5 : Gud levendegjorde ogsaa os med Christus , der vi vare dsde i Overtrcedelserne . Det menneskelige <b>Hjerte</b> forandrer da sin syndige Natur og antager Christendommens guddommelige Natur . To Mcend knnne gjsre os dette tydeligere: Paulus og Johannes . Paulus gik md i Husene i-Jerusalemi - Jerusalem og trak baade Mcend og <b>Kvinder</b> frem og overantvortede dem i Fcengsel , Ap . G . 8 , 3. C . 9 , 1 : Han fnps med Trusel og Mord mod Herrens Discivle . Hvilken Forbittrelse i det syndige <b>Hjerte</b> ! Da rorte Christendommen hans <b>Hjerte</b> og gjorde ham til et ganske andet Menneske . 1 Cor . 4 , 12. 13 : Overstjeldede velsigne vi ; forfulgte taale vi ; bespottede formane vi . 1 Cor . 13 , 4. 5 : <b>Kjcerligheden</b> er langmodig , er velvillig , forbittres ikke , bcerer ikke Nag . Hvorledes forstaae vi dette ? Paulus svarer os <b>selv</b> i Phil . 4 , 13 : Jeg formaaer Alt i Christo , som gjor mig stcerk . — Johannes . Vi lcefe Lue . 9 , 52 — 56. Det var Stolthed og Hevngjerrighet , ; saaledes bcerer det syndige <b>Hjerte</b> sig ad . Men hvor ganske anderledes senere ! 1 Joh . 4 , 11 : I Elskelige , har Gud saaledes elsket os ( hvorledes ? ) , da ere vi og skyldige at elske hverandre . Altsaa , det er den fsrste Virkning af Christendommen i Menneskets Indre , at den forandrer den syndige Natur , det syndige <b>Hjerte</b> til Lighed med Guds Vcesen . Iscer er det Stolthed og Hcevngjerrighed , hvori det syndige <b>Hjerte</b> fornemmelig viser sig , der ved Christendommen og kun ved Christendommen forvandles til Ad mygl ) ed og Sagtm odighed , saaledes som vi saae den hos Paulus og Johannes . Og endnu daglig see vi de samme Virkninger . Saaledes hos Bern og Voxne , naar deres Hjerter reres af Christendommen ; den legnagtige Natur forandres til guddommelig Natur , til <b>Sandhed</b> . De hcevne sig ikke , men tilgive , ligesom Gud . De ere ikke stolte , men deres Hjerter blive , ligesom Guds Syns <b>Hjerte</b> , sagtmodige o . s . v. Det nye Test . har mange forskjellige Udtryk for denne mcegtige Virkning af Christendommen ( Hvilken ? ) . Gjenfpdelse i fjerde Hovedstykke af den lille Catechismus . Den gamle Adam dser , og et nyt Menneske opstaaer i os . Den gamle Adam , Kjedet , det syndige <b>Hjerte</b> deer ; den , der deer , kan ikke mere udfere Noget , Synden har ingen Magt mere . Et nyt Menneske opstaaer , handler og virker ; altsaa det Modsatte . Christus lever i mig , siger Paulns , Gal . 2 , 20 , d . e . jeg har hans ( guddommelige ) Natur , hans Maade at tcenke , at tale , at handle paa . Men have vi saaledes annammet Christi Natur , saa blive vi derved deelagtige i den guddommelige Natur . Vi lcese 2 Pet , 1 , 3 — B . Har Du ogsaa det , som Apostelen i V . 5. 6 regner til denne guddommelige Natur ? nu faa gjer , hvad han forlanger ,  anvend al Flid . " Endnn et Udtryk : et Guds Barn , Rom . 8 , 14. 15. Til alt dette vil , som

<h4>2491</h4>giver Jesus sig tilkjende for hende , hvad han dengang endnu ikke havde gjort for nogen Jyde , og <b>Kvinden</b> troer hans Ord med stor Glcede . Hvoraf kunne vi see det ? Deraf , at hun lader sin Vandkrukke staae og gaaer til Staden og siger til Folkene der : Kommer , seer — . Jeg har Mad at « de , den I ikke kjende . Min Mad er , at jeg gjor hans Villie , som mig udsendte , og fuldkommer hans Gjerning . Disse Ord vise den Iver og Glcede , hvormed Jesus soger at gjsre os salige . Det er min Mad ; deri ligger , at det er Jesus en Glcede , en Vederkvegelse , en Fornpdenhed at gjpre Guds Villie . Men Guds Villie er netop , at Ingen stal fortabes , men at Alle stulle komme til Omvendelse , og denne Villie vilde Jesus til enhver Tid opfylde . Hvilken Trostab og <b>Kjcerlighed</b> ! Og saalebes er Jesus endnu : Din Hyrdestemme , o hvor mild den kalder hver , som farer vild . Hvorledes har Jesus kaldet og ssgt Dig ? Ved Daaben , ved Guds Ord . Du foer vild , d . e . Du spgte Frelse og Glcede og Salighet » i Synden : i Hcevngjerrighed , Logn , Stolthed o . s . v. Jeg har Mad at cede , den I ikke kjende ; thi den Salighed , at omvende en Synder , kjende kun Faa . Mon jeg vcere min Broders Vogter , sige Mange . Hvad betyde disse Ord her ? Da Judas vilde syge Treft hos Ippersieprcesterne , sagde de til ham : Hvad kommer det os ved ? see Du dertil . Saaledes ville vi ikke tale . Du kaldes en Christen og stal efterfolge Christus ; hvorledes og hvornaar gjor Du det da i det Punkt , hvorom vi her tale ? Og endnu et Skriftsted herom , som jeg ret vil lcegge Eder paa <b>Hjerte</b> , Jak . 5 , 19. 2 N : Bwdre , dersom Nogen iblandt Eder er faren vild fra <b>Sandheden</b> , og Nogen omvender ham , den vide , at hvo som omvender en Synder fra hans Veis Vildfarelse, han frelser en Sjcel fra Doden og stjuler Synders Mangfoldighed. Vi have <b>selv</b> hort og vide , at denne er sandclig den Verdens Frelser , den Christus . Nu havde de <b>selv</b> hprt det og vidste det , for kjendte de det kuu af <b>Kvindens</b> Fortcelling . I hore ogsaa fortcelle i Skolen om Jesus , og om at han er Verdens Frelser ; men I skulde <b>selv</b> hore det og <b>selv</b> vide og erkjeude det . Hvorledes steer det ? Naar I lade Eder sige af Jesus , at I ikke ere saaledes , som I skulde vcere for Gud , hverken i Tanker , Ord eller Gjerninger . Naar I lade Eder sige , at der endnu mangler Meget i , at Christus boer i Eder , d . e . at Eders Tanker , Ord og Gjerninger afspeile Jesu hellige Liv . Hvor fiudes hans Sagtmodighcd , Admyghed , Tjenstagtighed og Fromhed hos Eder ? Naar I fole dette , kj . 8. , saa erkjender fremdeles , at Christus kan give Eder Alt , hvad der fattes Eder . Han har elsket Eder indtil Doden og har ladet sit Liv for Eders Synder ; lad da denne <b>Kjcerlighed</b> vcekke Eders Gjenkjcerlighed , saa I beflitte Eder paa alt Godt ; med Pauli Ord : Christi <b>Kjcerlighed</b> tvinger os . Og dertil stal det ogsaa komme med Eder , paa det at , som Paulus siger i 1 Cor . 2 , 5 , Eders Tro ikke stal vcere i Mennestens Viisdom , men i Guds Kraft . Hvad mener Apostelen ? Kort sagt : Christus soger Eder , saa lader Eder da finde af ham . Kommer da , vi vil os samle om det Livsens Kildevceld, kommer Unge , kommer Gamle , kommer hver en mydig Sjcel !

<h4>2630</h4>ganske anderledes . <b>Kvinden</b> , som var greben i Hoer , Joh . 8 , fordommer han ikke , thi , Luc . 9 , 56 , Mennestens Son er ikke kommen at fordcerve Menneskers Sjcele , men at frelse dem ; ikke at domme Verden , men at Verden stal blive salig ved ham . Han siger : Den , som er syndelos iblandt Eder , kaste fyrst Stenen paa hende ; han fordommer hende ikke , men tilfoier saa ogsaa : Gak bort og synd ikke mere . Den samme Adfcerd viser han mod den Syge ved Bethesda Dam , Joh . 5 , 14 ; forst hjcelper han ham og derpaa formaner han : Synd ikke mere , at Dig ikke noget Vcerre stal vederfares . Som Frugt af denne <b>Kjcerlighed</b> til Gud og Menneskene fremtrcede iscer tvende Egenstaber hos Jesus , med Hensyn til hvilke han <b>selv</b> fremstiller sig som et Forvillede for os , nemlig Sagtmodighed og Yd mygl ) ed . Lcerer af mig , thi jeg er sagtmodig og ydmyg af HMet . lefus har aldrig hcevnet fig. Han fatte engang Disciplene irette , fordi de vilde lade Ild falde ned fra Himlen over en samaritanst By , og sagde , Luc . 9 , 55 : I vide ikke , af hvad Aand I ere ; thi Mennestens Son er ikke kommen at fordcerve Menneskers Sjcele , men at frelse dem . Han stjcendte ikke igjen , da han blev overstjcendt, men overlod Hcevnen til Gud , ja <b>selv</b> paa Korset bad han efter Spaadommens Ord i Es . 53 for Overtrcederne . — Jesus har fremdeles aldrig gjort Noget af Trods eller Lyst til forfcengelig 3 Ere . Han sogte ikke LEre af Mennesker , men af Gud ; og i Tro paa Gud og af <b>Kjcerlighed</b> til Menneskene fornedrede han sig <b>selv</b> og ventede kun sin Ovhoielse af Gud alene . I at vise Sagtmodighed og Idmyghed har han fornemmelig fremstillet sig <b>selv</b> som et Monster og Forvillede , fordi han vidste , hvor vanskelige disse tvende Dyder ere for det naturlige Menneskehjerte , og at de kun blive mulige at udove i Kraft af hans Forløsning , naar Mennesket trceder i Samfund med ham . — <b>Sandheden</b> gik ham over Alt . Han paaviser i Joh . 8 , 44 , at Djcevelen er Legnens Fader . Han siger ikke : Jeg er sanddru , men : Jeg er <b>Sandheden</b> , d . e . <b>Sandheden</b> er ham subjektiv , er hans Vcesen , er Eet med hans Vcesen . Men <b>Sandheden</b> er ham ikke blot den rigtige Tcenkning og Erkjendelse , men Overensstemmelse mellem hans og Guds Villie ; den fuldkomne Overensstemmelse mellem <b>Hjerte</b> , Ord og Gjerning indbyrdes og med Gud . Hvad han erkjender for at vcere guddommeligt , det siger og vil og tcenker og gjor han ; og aldrig « blev der funden Svig i hans Mund " , 1 Pet . 2 ' , 22. I Omgang med Andre har Herren ingen sceregne Regler fulgt . Han har glcedet sig med de Glade , som ved Bryllupet i Cana , og sprget med de Sorgende . Hans <b>Hjerte</b> har vceret aabent for Venskab , og yan kalder <b>selv</b> Lazarus for sin Ven : Lazarus , vor Ven , er sovet ind . Hans Samtaler med Kvin der udmcerke sig ved stor Fiinhed og Inderlighed ; man tcenke kun paa hans Samtale med den famaritanste <b>Kvinde</b> og hans Forhold til Lazarus ' s Sostre . Der findes heller ikke en eneste <b>Kvinde</b> i de evangeliske Fortcellinger , der var fjcndtlig sindet imod ham , ja <b>selv</b> Jerusalems Dottre grced over ham . Det var en <b>Kvinde</b> af Folket , der oploftede sin Rost og sagde : Salig er det Liv som bar Dig , og de Bryster , som Du diede . Det var <b>Kvinder</b> , der fulgte med ham og ssrgede for hans Fornodenheder , Luc . 8 , 1 - 3 ;

<h4>2874</h4>Vi gjennemgaae Indholdet af Capitlet . V . 1. En stor Forfplgelse begyndte ved Aaret 36 e . C . og blev 2 larfag i , at Giljrtftenbommen nu ogfaa udbredte sig udenfor 3 erufalem , der hidtil havde bæret © fucplabfen for Begivenhederne . De flygtende Christne forplantede nemlig Seeren til de andre Vyer i 3 iibæa og © amaria , og saaledes niaatte forfølgerne imod deres Stilte bjoelpe til at udbrede Christendommen , forn be netep bilbe undertrykke . Hvor <b>sandt</b> er bet dog , Ps . 2 , 4 , at ben , som boer i Himlen , l « r ad bem . Ifcer omtales Saulus , V . 3 , som gik md i £ > itfene og trak baade Mcend og <b>Kvinder</b> frem forat kaste dem i Fcengsel og , C . 26 , 10 , gav sin Stemme med til at bømme dem tildsde . Philip , som ncevnes i V . 5 , er ikke Apostelen , men Fattigforstanderen af dette Navn . Viis mig dette af V . 1. Apostlene bleve i Jerusalem og , V . 15 , meddeelte fyrst senere ben Helligaands Gave . 33. 12. I Samaria bar ben udstryede © æb strax Frugt , ligesom forhen ba Herren engang <b>selv</b> lcerte iblandt dem ; Joh . 4 , 35 betyder nemlig , at mebené der ellers er en Mellemtid metlem Übfceb og Host , faa falde de her sammen . — V . 13 og 17. I © amaria bel ) øt > ebe6 der ogfaa en t ; bre 33 æffelfe , ligesom paa den førfte Pintsedag og ved Helbredelsen af den Halte . Jesu og hans Vidners Kraft og Herlighed viste sig her i Tegn og Undergjerninger og i Meddelelsen af ben Helligaand , vidt forskjellige fra Trolddomskunster , et Forhold , der minber om Moses og de cegyptiste Troldmcend . Simon anfeer be to Apostle for sine Ligemcend og fremkommer derfor med sin Bon . 33. 18 og 19. Han meente , som Apostelen siger , at Gudsfrygt var en Vinding , Peters Svar er skarpt , V . 20. Guds Gave , nemlig Meddelelsen af den Helligaand , opnaaes ikke ved Penge ; nei , Gud skjenker den som en Naadegave . 33. 21. Kun den , hvis <b>Hjerte</b> er retstaffent , kan faae Deel i Christus . 33. 22. Derfor : Omvendelse og Ben om Tilgivelse . Candaccs Skatmcster . V . 27 viser os ret alle be Vanskeligheder, som denne Mand havde at overvinde forat komme inb i Guds Rige . Han var 1 ) en Mand fra LEthiopien ( Mcroe ) , det nuvcerende Nubien , hvor Dronningen pleiede at bære Navnet Sanbace . Hans Fcedreland var idetmindste 150 Mile fjernet fra Israel . Han var en Mand fra p , 3 ørbenB Ende " , Ap . Gj . 1 , 8. — 2 ) En LEthi- ter , en Neger , nedstammende fra Cham , Faderen til Canaan , forn blev forbandet , 1 Mos . 9 , 25. Hvor fjernt en Saadan stod fra Guds Folk , see vi ogfaa af 4 Mos . 12 , 1. — Han var tillige 3 ) en fornem og mægtig Mand , og 4 ) en Mand , hvem Dronningens Skatte vare betroede , han var Skat mester . Men Herren siger i Matth . 19 , 23. 24 : Sandelig siger jeg Eder , at en Rig kommer vanskelig ind i Himmeriges Rige ; det er lettere , at en Kameel gaaer igjrnnem et Naalepie, end at en Rig kommer ind i Guds Rige . Alle disse Hindringer blive overvundne . V . 27 : Han var kommen til Jerusalem forat tilbede . Den guddommelige <b>Kjcerlighed</b> n . , 9

<h4>2976</h4>hvilken Inderlighet , elskede han ikke Herren — naar han kun kunde have den Glcede , at Ipderne bleve frelste . Hvilken KjVrlighed ! — 2 Cor . 2 , 4 : I megen Kummer og med beklemt <b>Hjerte</b> skrev jeg Eder til under mange Taarer , ikke at I skulde blive bedrede , men paa det at I skulde kjende den <b>Kjcerlighed</b> , som jeg ovcrslydigen har til Eder . — 1 Cor . 4 , 14. 16 : Jeg paaminder Eder , som mine elskelige Born ; jeg avlede Eder i Christo Jesu ved Evangelium . — 1 Thess . 2 , 7. 8 : Vi omgikkes mildt med Eder ; som en Amme pleier sine Born , saaledes ere vi as inderlig <b>Kjcerlighed</b> til Eder villige til at dele med Eder ikke alene Guds Evangelium , men ogsaa vort eget Liv , fordi I ere blevne os elskelige . — Altsaa ligesaa omhyggelig , betcenksom , om , <b>kjcerlig</b> og mild som en <b>Moder</b> , der giver sit Barn <b>Die</b> ; kan der vel tcentes noqet fmukkere Billede ? - Til de Wldste i Menigheden i Ephesus siger han , Ap . Gj . 20 , 31 : Konmwr ihu , at jeg har ikle aftadt i tre Aar , Nat og Dag , at paaminde enhver as Eder med Taarer . 1 Thess . 3 , 10 : Vi bede indstcendigen , Nat og Dag , at vi maatte see Eders Ansigt og raade Bod paa Eders Troes Mangler . Og med hvilken Alvor og <b>Kjcerlighcd</b> anbefaler han ikke Menighedernes Vel til de i Milet forsamlede ZEldste , Ap . Gj . 20 , 28 : Saa giver Agt paa Eder <b>selv</b> og den ganske Hjord , iblandt hvilken den Helligaand satte Eder som Tilsynsmcend, at vogte Guds Menighed , hvilken han forhvervede med sit eget Blod . * ) — Med den samme <b>Kjcerlighed</b> , som han viste mod hele

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014110428003'target=_>Francke, August Hermann, 1879, Huus-Postil, eller Prædikener paa alle Søn- og Fest-Dage i Aaret</a><h4>175</h4>eder , som man ikke kan formane til Aarvaagenhed paa den Maade , som Christus har formanet sine kjcere Disciple dertil , men som man endnu forst maa raabe til : ( Eph . 5 , 14. ) „ Vaagn op , du som sover , og stat op fra de Dsde , og Christus stal lyse for dig . " Hvo dette horer , den gaa i sig <b>selv</b> , og betomke , om han vel nogensinde i sin ganske Livstid har fundet sig ret opvakt til usie at underssge sin ganske Omgjcrngelse og indvortes Hjertens-Bestaffenhed efter Guds Ord , for at blive vis paa , om han i saadan Tilstand og kunde forsikre sig om Guds Naade , og med Vished haabe at blive evig salig ? Mangen En , naar han kun lidet vil overveie det , skal ganske let overbevises i sin Samvittighed om , at han hidindtil har ladet det blive derved , at han i sin Barndom er bleven holdt til Skole , er derpaa ganen til den hellige Nadvere , har besogt Prcediken , har nogenledes gjort sin Morgen- og Aftenbon , men aldrig bekymret sig om en retskaffen Alvorlighed i sin Christendom , og hvorledes han ial sin Omgjcrugelse maatte have Gud for <b>Oie</b> og i <b>Hjerte</b> . Det Allerbedrsveligste derved er endda det , at man har havt den Indbildning , at man allerede er en god Christen , i hvilken Mening mangen En staar saa sast , at han vel skulde svcerge paa , at , om han nu dsde , saa for hans SM af Munden lige til Himmelen ; og om Nogen sagde til ham : Dn er endnn ingen ret Christen, saa skulde han vel blive saare vred paa den samme . Skulde nu Gud velsigne dette Ord hos Nogen , og han kom til Overbevisning om , at han hidindtil har vwret en saadan , som nu er bleven bestreden , saa agte han det for vist , at Herren <b>selv</b> af stor <b>Kjcrrlighed</b> , for at redde hans Sjoel fra Fordærvelse, har ladet dette Ord raabe til ham : Vaagn op , vaagn op , du som sover , og stat op fra de Dode , og Christus stal lyse for dig . Den , som saa vidt har velsignet Ordet hos ham , at han nu allerede erkjender , at han hidindtil har vwret saadan En , nemlig en Sovende og Dsd , han stal og fremdeles lade det vcere kraftigt paa ham , saafremt han kun ikke imodstaar hans Aands Virkning. Men der ere endnu Andre , som have faaet et Slag paa deres <b>Hjerte</b> , og ere blevne saa vidt opvakte ved Ordet , at de have kjendt , at det stod ikke ret til med dem , og at de nsdvendig maatte blive anderledes . Saadanne har jeg Aarsag at tilraabe af 1 Cor . 15 , 34 : Vaagner dog engang ret op , bliver dog engang ret crdrue og opvakte , og fynder ikke . Thi de ere lige de Slumrende, og som de , der have Mnene halv aabne , og hverken ret vaage , ei heller ret sove . De ere hverken kolde eller varme ; men vide I ikke , at Christus siger : ( Aab . 3 , 16. ) „ Efterdi du er lunken , og hverken kold eller varm , stal jeg udspy dig af min Mund " . Hvorlcrnge ville I gaa hen i saadan en fordelt Tilstand ? Bryder dog frist igjennem ! Christus vil forlene eder sin Naade og Kraft dertil, at I maa komme til en <b>sand</b> Forandring , og en grundig Forbedring i eders Levnet . Tvinger eder <b>selv</b> til Bsnnen , skynder og driver eders <b>Hjerte</b> dertil , ved det I forestille eder Fornsdenheden deri , paadet I ikke skulle blive vante til saadan en slumrende Tilstand , som er i hoieste Maade stadelig for eders Sjcrle . Paa den fsrste Ssndag i Advent ere I paany blevne opvakte ; idag er det den anden , da maa I endnn bedre trcrnge ind , og gribe Sagen an med stsrre Alvorlighed, ellers gaar igjen Tiden saaledes hen fra en Sondag til en anden , indtil Kirkeaciret lsber tilende , og I blive dog ikke ret anderledes . Derfor

<h4>352</h4>scmfremt det stal findes stikket til at smage og see Herrens Venlighed , og derved optcrndes til den forste <b>Kjcrrlighed</b> til ham . Man skulde vel mene , at man hos Mange skulde trcrffe denne fyrste <b>Kjcerlighed</b> til den Herre lesum , siden de ndvurtes intet lastværdigt Levnet fore , men meget mere elske den udvortes Mrbarhed og ogsaa med Munden foregive en <b>Kjcrrlighed</b> til den Herre lesum ; men naar man beskuer det i Grunden , finder man , at de falsteligen indbilde sig <b>selv</b> , at de elske den Herre lesum , hvoraf de dog intet andet have end det blotte Skin , men ncrgte den sande Kraft , og hverken have seet eller kjendt denne , saa at de kunne vide , hvordan den SM er tilmode , som den Hellig-Acmd har virket den forste <b>Kjcrrlighed</b> i til den Herre lesum . Saa blive da aleue de Sjcrle tilbage , som Gud kraftig har rort ved sit Ord , og som ogsaa have givet denne Rorelse saa vidt Rum hos sig , at de af Hjertet have omveudt sig til Gud , og ere komne til den sande og levende Tro til lesum Christum ; ligesom Joseph , Maria , Simeon og Prophetinden Anna ( ihvorvel uden Tvivl i forskjellig Grad ) have troet paa den Herre lesnm , og kjendt ham . Hos Saadanne maa man ogsaa nnomstnnder soge den forste <b>Kjcrrlighed</b> til den Herre lesum ; thi denne <b>Kjcrrlighed</b> er en Frngt af Troen , men der kan ingen <b>sand</b> Tro vcrre , hvor der ingen <b>sand</b> Omvendelse er . Er der nu ingen Tro , hvor kan da <b>Kjcrrligheden</b> til den Herre lesnm vcrre af den rette Art ? Vel kan det ske , at En , som Gud isandhed har grebet formedelst sin Naade , endnu ikke er kommen til nogen Troes Frimodighed , eller Folelse og Fornemmelse af den ; men der maa dog vcrre en <b>sand</b> Lcrngsel efter den Herre lesnm , og en Venten , under alvorlig Bon og Suk , paa en klarere Forsikring om hans Salighed i Christo Jesu . Saadanne Sjcrle ere da i visse Henseender dem lige , der ventede paa Forlosningen i Jerusalem , hvorom der staar i vor Text . Ja de ere de Disciple lige , der reiste til Emaus ( Luc . 24 , 32. ) , hvis Hjerter brcendte i dem , og dog ikke vidste , at det var den Herre Jesus , der vandrede med dem . Saaledes gaar det Mange , der her grcrde over sine Synder, og ingen Trsst kunne komme til ; ligesom ogsaa Mange , der klage kun over deres Elendighed og Fordcrrvelse , og ingen Naades Fornemmelse kunne komme til . Disse kan man da ikke udelukke fra den forste <b>Kjcerlighed</b> til den Herre lesum omeudskjont de <b>selv</b> allermindst holde sig for Saadanne , som elske den Herre lesum . De lede efter ham ved Graven , som Maria , og vide ikke , at han , alle bodsirrdige Sjceles trofaste Elsker , er dem saa ncer . Betragter jeg ellers vor Text , og bctcenker , hvem der vel paa det smmeste har elsket vor Herre Jesus , saa var det udentvivl hans Moders Maricr omme <b>Hjerte</b> . O hvad maatte der have ganet for sig ! hvilke Kjcerlig- Heds-Flammer maa der ikke have vcrret til dette Barn , som hun havde fsdt , og om hvilket der var sagt hende saa store Ting ! Altsaa sige vi da hellerikke udeu Gruud , at <b>Kjcrrligheden</b> til deu Herre lesum er allersmmest hos en saadllu Sjcrl , der aaudeliguis er bleveu Christi <b>Moder</b> , saaledes , og i den Forstand , som Christus <b>selv</b> taler derom ( Matth , 12 , 49. 50 ) . Men for Kortheds Skyld ville vi for denne Gang ikke tale mere derom , helst siden det ogsaa vil blive klart af det Fslgende .

<h4>1100</h4>vor Snliggjsrelse . Her stulle vi nu forestille os i vort Sind , hvorledes dette Budstab , som Engelen bringer Maria , forsaavidt som det augnar den himmelske Faders uendelige <b>Kjcrrlighed</b> , ligesaavel vedkommer os som Maria . Dersom vi ville ret betcrnke det , saa skulde vi hverken blive staaende ved Maria, eller ved Engelen Gabriel , eller ved nogen anden Omstændighed i Sagen, men alene tage til <b>Hjerte</b> , og alene glcede os ved det , som er den dybeste og allerinderste Grund i denue hele Sag , nemlig Guds og vor himmelske Faders evige <b>Kjcrrlighed</b> , af hvilken saavel vor ganske Saliggørelse som deune trsstefnlde Forkyndelse er kommen . Staa stille , kjcrre Sjcrl ! og betcrnk meget dybt dette Textens Ord : af Ond ; pandet du der man gaa gcmste ind i Guds <b>Hjerte</b> , og med Troens Mne efter al diu Lyst beskue din Skabers og Faders evige Kærligheds uendelige og ganske vidunderlige Skatte ; nt du der mna se og betragte , hvorledes den levende Guds Faderhjerte har bevcrget sig imod dig i den hoieste og reneste <b>Kjcrrlighed</b> ; at du der maa hore , hvorledes han for din Skyld , og pandet han igjen kunde forbarme sig over dig og indtage dig ial sin Saligheds Skjod , har sendt Gabriel til Maria , at forkynde hende , at hans Kjcrrligheds Son skulde fodes som et <b>sandt</b> Menneske af hende , og derpaa vcrre al Verdeus og din Frelser og Saliggjsrer . Ind , ind i Guds <b>Hjerte</b> , o SM ! her er intet fornodent uden Troen , der i stor Enfoldighed beskuer denne evige Kjcrrligheds Kilde , og oser det evige Livs Vand dernf med Gwde . Vi ville dn ogsnn dennegang vedblive at beskue denne Kjcrrligheds Kilde , og at ose Livsens Vand deraf for vor Sjcel ; hvorfor vi dn forbigna de Tids- , Persons- og Stedsomstcrndigheder , hvilke ellers nsdvendigen Horte med til Sagens Fortælling , og meget mere betragte de Ord , som Engelen talte , da han kom til Maria . „ Hil vcrre dig " , sagde han , „ du Benaadede ; Herren er med dig ; du Velsignede iblcmdt <b>Kvinder</b> . " Dette var hans Hilsen, hvorved vi stedse maa tcrnke tilbage paa det , vi forst have rettet vore Tanker paa , nemlig : at Engelen har vcrret sendt af Gud ; thi snaledes maa vi nu ogsaa anse denne hans Hilsen , ikke som den var bragt Maria fra Engelen, men som bragt hende ved Engelen fra Gud <b>selv</b> . Engelen talede som i Guds Sted . Derfor have vi af hans Ord meget mere at erkjende Guds vor himmelske Faders , end hans ( nemlig Engelens ) eget <b>Hjerte</b> . Og ligesom nu Marin vel var den , af hvilken Christus skulde fodes , men Christus og hans Forlosning dog ikke vedkom Maria alene , men alle Mennesker , snn have vi heller ikke nodig i Engelens Hilsen at bemcrrke Guds <b>Kjcrrlighed</b> til Marin nleue , meu til det ganske menneskelige Kjon . Han siger til Maria : „ Hil vcrre dig " , eller , som det lyder i det Grcrste , „ glad dig dn Benaadede ; " men hvad er det andet end som om den himmelske Faders <b>Hjerte</b> nabnede sig for alle Mennesker , der ligge under Syndens Elendighed , og sagde til dem : Glcrder eder , I arme Syndere , thi hidindtil trykker Vreden og Unnnden eder for Syndens Skyld , men I ere hsit benaadede , — nn gaar Naaden op

<h4>2734</h4>Ak ! lader mig dog tale med eder , mine Elskelige , med al Frimodighed og som for Gnds Ansigt . Hvor ganske elendig er dog endnn Mangens Tilstand! Verdens <b>Kjcerlighed</b> er saaet saa tykt iblandt eder , og har fra Ungdommen af slaaet saa dybe og storke Rsdder i eders Hjerter , at , om man end finder Mange , der fynes gjerne at hore Guds Ord og villige at antage det , saa kan dog saadcint , naar man uudersoger det i Grunden , ikke kaldes det retskafne Vasen , som er i lesn . Der er ikke den rette Alvorlighed , ikke den sande Nidkja-rhed for Guds 3 Erc , ikke den saa dyrt anbefalede Strcrben efter Helliggjorelse , ikke den Gud behagelige , bramdende Lne af Christi Kærlighed , der er , som et Blns og en brændende Ild i hans sande Disciples Hjerter . Leder man hos Mangenen , der ellers er en flittig Tilhorer , efter disse rette Kjendetegn paa den sonlige Udkaarelse hos Gud , saa finder man intet Spor dertil . Men det finder man vel , at de ved deres antagne Christendoms Skin ere ganske fulde af Vantro og Næringssorg , eller fore deres Levnet hjemme paa gammel Vis , elske Drik og Selskab , eller have andre saadanne Ting ved sig , der umulig kan beståa med en <b>sand</b> Hjertens Forandring . Saadanne staa da eller sidde i Kirken , og hore paa Alt ; men man maa tramge saa skarpt ind med Ordet paa deres Hjerter , som man end vil , saa antage de det dog knn udvortes , og , fordi de derhos have den Indbildning , at det allerede er godt fat med dem , saa lade de ikke Ordet komme saa dybt ind i deres <b>Hjertes</b> inderste Grund , at de kunne komme til ret Eftertanke om , hvor stor en Forandring deres ganske Vcrsen endnu behovede , saafremt Christus skulde erkjende dem for Sine . Saalamge nu eders Tilstand er saadan , saa kunne I ikke blive hjulpne , om I end altid ere tilstede og hore paa Prcrdikenen . Her sidder En , og tcenker paa sine Processer ; hist sidder en Anden og grunder paa , hvorledes han kan saa sit timelige Gods formeret ; hist en Anden , der plager sig med Næringssorger ; hist En igjen , hvis Sind endnu er adspredt , og endnu ikke bleven ret cedru af sit onde Selskab og af sin Sus og Dus , hvori han lever ; her og der sidder et <b>Fruentimmer</b> , der er besnæret af daarlig Forfængelighed i Klædedragt og i sit Sinds Hoffardighed ; Andre ere saa indtagne af allehaande jordiske og verdslige Lyster , at Ordet derfor ikke kan gaa til Hjertet ; Andre ere saa kolde og dovne i deres Vcesen , at de sidde og sove og slumre under Prcrdikenen , og naar den er endt , saa vide de ikke , hvad der er prcediket . Ja , hvem der lod sig forblinde af Tilhorernes Mamgde , han maatte tcenke , Guds Rige grsnnedes og blomstrede iblandt eder . Men , o ! hvor Faa ere der , der med tilbsrlig Opmærksomhed og med Tilegnelse paa sig <b>selv</b> hore paa Ordet! Hvor Faa ere der iblandt eder , som ikke forhindre sig <b>selv</b> , saa at Evangelii

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112200022'target=_>, 1870, Bibelen, eller Den hellige Skrift</a><h4>2254</h4>gang det samme som at overgive dem til Djcevelen , 1 Cor . 5 , 5. 1 Tim . 1 , 20.  Men en saadan Forbandelse synes dog kun lidet overensstemmende med den apostolisse Sagtmodigbed , da vi jo meget mere stulle enste , at Alle maa blive salige , og ikke en eneste gaa fortabt . Dette er ogsaa <b>sandt</b> , naar vi see hen til Menneskene . Thi Alles Frelse uden Undtagelse bliver lagt os paa <b>Hjerte</b> af Gud , ligesom jo ogsaa Christus led for den ganske Verdens Synder . Men fromme Hjerter blive undertiden hcevede op over dette Hensyn til Menneskene , idet de blot have Guds Wre og Christi Rige for <b>Die</b> . Thi da Guds Wre siaaer langt hsiere end Menneskenes Frelse , saa stal den ogsaa drage os i Hsiden , saa at vi elske og soge den fremfor alt Andet . Herved rettes de Troendes Strceben ganske og aldeles paa Guds Forherligelse , saa de glemme Verden og Menneskene , og heller enste , at den ganske Verden gik under , end at Guds Wre skulde lide det mindste Afbrcek . Cndvidere , naar vi sammenligne eet eller nogle faa Mennesker med Guds hele Menigbed, hvor meget maa da ikke hiin veie op hine faa ! Det er en grusom Barmhjertighed at bekymre sig mere om et Menneske end om den hele Menighed . Paa den ene Side seer jeg Herrens Faar i Fare , paa den anden Side seer jeg Ulven drives af Satan til at gaae ud paa Rov ; hvorledes ? skulde nu ikte min <b>Kjcerlighed</b> til Menigheden tåge saa aldeles min Omsorg i Beslag , at jeg maatte enste , den blev frelst , <b>selv</b> om det kosteoe Ulvens Undergang ? Hermed onster jeg dog ikke Nogens Fordsmmelse , men min Kjaerlighed til og Omhu for Menigheden sattter " mig ligesom udover mig <b>selv</b> , saa at jeg ikke ncrrer nogen anden Tanke . Derfor er heller ikke den nogen oegte Hyrde

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009082103022'target=_>, 1859, Christelig Psalmebog</a><h4>672</h4>nit S.nd , At jeg ret mig til dig vender , At dit Legems Skat kan faae Rum udi mit <b>Hjertes</b> Vraae , At jeg ret din <b>Sandhed</b> kjender , At jeg , fri fra Syndens Stand , Dig , min Gnd , behage kan . 9. Lifligt er din Boligs Seede , Fnldt af Naade , Liv og Godt ; Fred , Retfcerdighed og Glade Flyter til os fra dit Slot , Gud mit <b>Hjertes</b> Grund forlanger . At jeg gjennem Tidens Vee Maa med starve <b>Oine</b> see , Hvor din hsie Bolig pranger , Da i dig saa inderlig Liv og Sjel forlyster sig . 10. Vel da den , sin Sjel forlyster Daglig iat prise Gud , Paa hans Naade sig fortroster , Gaacr frimodig ind og ud , Som dig holder for sin Klippe , Som i Dyd og Stilhed vil Deres Levnet bringe til , Som vil aldrig lesum slippe . Hvo saaledes vandrer frem , Han stal finde Himlens Hjem . 11. Saadan ret tilsinds at blive , Gjor min Jesu <b>Kjarlighed</b> , Som ham <b>selv</b> har kunnet drive Fra sin Himmel til os ned , Ol hvor kan hans Dsd os glade , O ! hvor fryder os hans Ord , At vi paa den usle Jord Seer Guds skjonne Himmelsade, Og af Livets Strsm og Aae , Stundom nogle Draabcr faae . 12. Lukker alle hoie Porte Op i Verdens vide Land , Fiendens Frygt og Magt er borte , Onster Lytte alle Mand , Wrens Konge af tet Hsie Kommer i sin Hcrrepragt , Drager til os ind med Magt , Derfor Alle sig bcmsie , At hans Priis kan blive stor Paa den ganste Verdens Jord . 18. Lover Alle og tilbeder Denne slarke Herre Gud , Som sit Riges Grcrndser breder Ved sin Naade daglig ud ; Han er Gud af ham vi haver , Hvad vi er og bar behov , Hannem vcere evig Lov Kor hans mange store Gaver , Gud , som er saa mild og blid . Vare Lov til evig Tid .

<h4>2027</h4>bevarer Fra det forste Oieblik , Da jeg Sjel og Aande fik . Ja til denne Time sparet ; Alt tilsidst en Ende faar . Men Guds <b>Kjarlighed</b> bestaar . 3. Sin eenbaarne Son han itte Sparede , men drcebes lod , At han mig fra Satans Strikte Kunde frelfe ved hans Blod ; O ! hvorledes stal jeg kunne Med min Aand og svage Sind , Udi Morkel viklet ind , Saadan Naades Dyb udgrunde ; Alt tilsidst en Ende faar . Men Guds <b>Kjarlighed</b> bestaar . 4. Han sin Aand mig gi ' er til Eie I sit Ord , ved hvilket han Forer paa de trange Veie Mig til det forgjatted ' Land : Aanden og mit <b>Hjerte</b> fryder Med en <b>sand</b> og salig Tro , Som forstyrrer Satans Ro , Ia felv Dodens Magt nedbryder ; Alt tilsidst en Ende faar , Men Guds <b>Kjarlighed</b> bestaar . 5. Alt det , som min Sjel kan gavne , Han anordnet har med Flid , Lader ei mit Legem ' savne Raad og Hjalp i Nodens Tid : Naar min Kraft , min Magt , min Styrte Intet magter . Intet lan , Tager Guo mig ved min Haand Og begynder <b>selv</b> at virke ; Alt tilsidst en Ende faar , Men Guds <b>Kjarlighed</b> bestaar . 6. Jorden med Alt hvad den eier , Himlen med dens ganste Har , Hvor mit <b>Oie</b> sig hendreier , Alt til Gode dannet er ; Dyr og Urter , Scedens Grode , Skove , Fjelde , Mart og Lund , Ferste Vand ' med Havets Grund Stulle ratte mig min Fode ; Alt tilsidst en Ende faar , Men Guds <b>Kjarlighed</b> bestaar . 7. Naar jeg sover , Herren holder Vaagent <b>Oie</b> over mig , Og ved hver Dags Morgen volder , At hans Gunst fornyer sig : Havde Gud ei tåget vare . Og mig trolig staaet bi . Var jeg ikke sluppet fri Af saamcgen Nod og Fore ; Alt tilsidst en Ende faar , Men Guds <b>Kjarlighed</b> bestaar . 8. Hvad har Satan dog udgrundet Mange Plager, mangen Slrcif ! leg har dog til Intet sundet ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008112403025'target=_>, 1861, Bibelen, eller Den hellige Skrift</a><h4>584</h4>pende og lcrngselfuldt ventende Bsrn ere stedte ; idet han ikke alene opflam < mer deres <b>Hjerte</b> til Bsn , og retter deres <b>Blik</b> paa det rette Maal , men ogsaa , naar det desuagtet tåger feil , og deres brcendende Andagt truer med at kjslne , taler til Gud udaf dem <b>selv</b> med uudsigelige Sukke , der ikke kunne udtrykkes i svage , syndige McnnesserS Ord . Falskt er det , og Vold paa Ter . tens Ord , naar man forklarer denne Aandens Sukken : « han bringer os til at sukke . " At den Helligaand , som <b>selv</b> er Gud , paa denne Maade taler til og paakalder Gud , bor ikke fore < komme os mere besynderlige end at Christus . vor Appersteprast , beder for os , da han var ber paa Jorden , og endnu den Dag i Dag ved Guds hoi » re Haand ; meget mere maa denne sto » re , her afslorede Hemmelighed , at Guds Aand saaledes bliver Et med vor Aand , opvcekke os til beundrende Tilbedelse af Guds Kjaerlighed og Nedladelse . Menneskets Personlighed er ikke noget forbigaaende Skin , oplsser sig ikke i det almindelige Liv ; meget mere lever han kun da ' i <b>Sandhed</b> et Aandens Liv , naar Guds personlige Nand , han , som fra Evighed af forener Faderen med Sonnen , Gud med sin er Sjcrlen i hans Liv , er Gud i ham . Fra Evighed af er Gud den almcegtige, hellige <b>Kjcerlighcd</b> , og , for at va > re det , behovede han ikke Verden , var i denne sin Egenssab ikke afhamgig af den og dens Tilværelse ; den evig ham lige , fuldkomne Gjenstand for hans <b>Kjcerlighed</b> er meget mere Sonnen , ved hvem han har stabt Verden til sin Forherligelse i evig <b>Kjcerlighed</b> . Men de tyende Personer , Faderen og Ssnnen, vilde < Evighed nsdvendig danne en uoplsst Modscetning , hvis ' ikke den frie , hellige Kjarligbeds personlige Aand , der udgaaer af begge , evig forenede dem ; og det , som Gud har skabt , vilde blive fremmed for ham og skille sig fra hans Samfund , hvis ikke Sjcelen i alt Liv deri var den , ved hvem Fa < deren og Ssnnen evig ere Ct . Ligesom

<h4>4743</h4>Egtefolk til Pligt , ssulle Alle lcrgge Vind vaa ; den ydmyge <b>Kjcerlighed</b> giver Alle eet Eind , lader os fole Andres Nsd som vor egen , vender sig iscer gjerne til Brodrene , fordi den der sinder Gjensvar , og er stedse fuld af den inderligste Hjertens Deeltagelse . 1. . V . 9. ' Fordi vi <b>selv</b> ere kaldede til Troen paa Christus , og dermed til engang at opnaae Velsignelsen , det evige Livs frie , « fortjente ' Naadesfor » jcrttelse , stulle vi ogsaa istedetfor For » bandelse altid onste Andre idel Vel < signelse . 2. V . 10. D . e . som i <b>Sandhed</b> vil glcede sig i sir Liv . ' 3. V.12 . Ps . 34. 13 — 17. Ordret :  Thi Herrens Dine ( ere rettede ) Paa de Retfcerdige , og hans Dren " paa deres Bon ; men Herrens Ansigt ( er rettet ) paa dem , som gjsre Ondt . " De forste Ord betegne Guds inderlige , levende Attraa efter at staac og opfylde alle deres Nonner ; ved de sidste OrdcrTillceggetiPsalmen :  for at udrydde deres Ihukommelse a f Ior » den " udeladt , dog saaledes . at Laserne i Tankerne stulle ' supplere dem . Guds <b>Die</b> og Dre udtrykler hans inderlige Deeltagelse , Ansigtet Magten afbans Vrede ; ibvorvel ogsaa dette Ord kan bruges om hans naadige Beskuelse . 4 Mos . 6 , 24 - 26. 4. V . 13. I Reglen staber vort eget <b>Hjertes</b> Frygt , Klnnmodigbed , Uvlshed og Ureenhed de fleste Lidelser . Tragter man stille og fast efter det Gode , saa nyder man fordetmeste og . saa i det Udvortes Guds synderlige Beskyttelse .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111903002'target=_>, 1856, Prædikener til hver Søn- og Festdag i Aaret</a><h4>1455</h4>dig : Er du fra din Barndoms Dage af blevcn stamnde i din Daabspagl, sta du hm dit hele Liv igjennem forsaget Djevelen og alt hans Vesen og alle hans Gjerninger , hådet Synde » i alle dens Skikkelser mco et hjerteligt Had og derimod bygget med hele dit <b>Hjertes</b> Haab og Tillid i eenfoldig Tro paa den treenige Gud Fader , Son og Hellig- Aand ? Kan du i Hjertets Oprigtighed for Herrens Aasyn svare ja hertil , sta er til din Guds Lov og Tak Naaden , stm i Daaben blev dig til Deel , kraftig i dig , og du cr et Guds Barn , sta vandrer du paa Fredens Vei , paa hvilken Gud bevare dig , sta du aldrig vender om fra den . Men hcrvcd maa jeg dog bede Eder , kjen Venner , at I ste til , at I ittc bedrage Ever <b>selv</b> i sta vigtig en Sag . Og nnar jeg stal sige min Mening , da maa jeg efter den Erfaring , jeg hidindtil har vundet , nen hoilig Frygt for , at neppe Nogen af Eder er blcven stamnde i sin Daabsnaade . Ja , det er en strgclig Erfaring , og den vidner om c » stor Vrede baade hos Unge og Gamle , om desto storre en Brode , jo eldre du er . Eller er itte det Brode , M du hm forodct din Arv , den barnlige Tro og <b>Kjerlighed</b> og Lydighet » , at du modvilligcn har forladt din Fadrrs Huus og milde Tjeneste og kastet dig i Syndens Favn og cr blcven Djevelens Trel , at du har hamret og foragtet din Herres og Frelsers Blod , der vilde rense dig nf al Synd , at du hnr modstamt den Hellig-Aand indtil denne Dag ? O Venner ! det var en megtig Guds Kraft , der greb os i Daaben ; derfor maa ogsta den Syndens Magt ven redsom stor , dcr kan overdove og tilintetgjore hiin , sta at dcn neppe kjendes mere . In maastee er her Mange , som neppe vide af , at nogcnstndc Guds Kraft fra Daabs - Naadcn af har rott sig i dem , der lige fm den spedeste Barndom af vorte op i Synd , og i hvis Sjele . Liv lun den Forskjel ssndcs , at jo eldre de bleve , desto ivrigere tjente de Djevelen stm sin Herre . Delte hores vel stretteligt , men tro mig , at sta gaacr det maastee endog mcd de Fleste blandt os . Men hvornf kommer det , siger du mcd Forferdelse , da Barnehjertet dog itte strm er sta fanget af Syndens fule Lyster ? Ja hvoraf kommer det , siig mig det , du Fader og <b>Moder</b> , du , som sattes til at vogte om Guddomslivct i Barnehjertet , til at opdrage dine Born i Herrens Tugt og Formaning, til at len dem at clste og i barnlig <b>Kjerlighed</b> adlyde den Gud , til hvis Born dc i Daaben cre antagne ? Har du opfyldt dette dit Kald ? Eller har du sta langt fra dette , at du meget mere har forarget dine Smaa med Ord og Erempel , og stalcdcs heller fort dem bort fra Hcrrcn end hen til Ham ? Da vil jeg sige dig et Ord , et alvorligt Ord , men som er talt af <b>Sandhedens</b> Mund : „ Det var dog bedre , at der var hengt en Msllesteen om din Hals , og du var nedsenket

<h4>2206</h4>lov , saa at Talen kunde fores om og Forskjellen udhceves imellem det Legale , det er det med den borgerlige Lov Stemmende , og det Moralske, det af Gud Fordrede . Det er dette Sidste , som sta ofte blev oversett , og hvorved Forskjellen opstod imellem en <b>sand</b> og falsk Christendom; det er dette , der lod Pharisceerne have sin Lon borte , der lod den fattige Enkes Skjcerv veie mere end de Riges store Gaver , og som paa vor Regnstabsdag vil lade mangen Gjerning , stm lovpristes i Livet, Me veie en Avne i Retfcerdighedens Vcegtstaal . Det er heri det klarligen viser sig , at Christendommen , denne guddommelige Lan , staaer i et forkeert Lys hos sta Mange , der i samme ste en blot Rettesnor for vore Handlinger i Almindelighed og ei det store Dannelstsmiddel i Herrens Haand , hvorved et forbedret , et i <b>Sandhed</b> Gud hengivent <b>Hjerte</b> skulde vindes , og enhver god Handling kun vare en umiddelbar Frugt af , hvad der rotte sig i vort Bryst . Staaer den nemlig for os som det store Kjcerlighedsbeviis , Gud har givet Verden , ste vi i den Vidner om det Fadersind , hvormed vi ere omfattede , ste vi den med Troens <b>Oie</b> , sta at hvad den forkynder om Synd og Naade , om Frelse og Salighcd ene i Guds eenbaarne Son , er blevet os klart , da maa den nodvcndig bcere sine Frugter , og disse henfore vi deels til det Indre hos os , stm kun det guddommelige <b>Oie</b> ster , deels til det Idre i Livet , til Handlingerne , som ere aabenbare og Verden bekjendle . Ved det Indre forstaae vi de Folelser , der stulle fremkalde vore Handlinger; og fremtråder Christendommen for os saaledes , som Pi nylig have ncevnt , maae disse Folelser blive Idmyghed og <b>Kjerlighed</b> , Tillid og Hengivenhed i Guds Villie og en Afsty for , hvad der er Ondt , hvad der modarbeider den Plan , som Gud vil have fremmet med os . Det er Christendommen , der nedstacer det stolte vg hovmodige Sind ' ved at vise , at Alt , hvad der sijcenkes os , er Naade ; det er den , der bringer vs til at erkjende <b>Sandheden</b> af , hvad Apostelen siger , at vi have syndet og fattes den Ros , vi stulle have for Gud ; det er endelig den , der lader os efter Prevelse af os <b>selv</b> og vore Handlinger, bede med oprigtigt <b>Hjerte</b> : „ Herre ! gak ikke til Doms med os , men var os Synden nacrdig ! " Og nåar den viser os Guds Naade , som os er vedcrfaret ved Christi Komme til Verden , viser os det hele Forsoningsvcerk , viser os Gud ci blot som den Hellige og Retfceldige , men og stm den Barmhjertige , og nåar der indbydes i vor Frelsers Navn ! ) : „ Kommer hid til Mig Alle , stm arbeide og ere bestande , og Jeg vil give Eder Hvile " , da maa vel Grundfolelstn i vor Barm

<h4>2362</h4>som isse er i <b>Kjcerlighed</b> , er isse i Gud , og Gud ei i ham . Vidste vi det end isse , at hver den , stm siger , at han ikke bar Synd , er en Logner, og der er Me <b>Sandhed</b> i ham ; vidste vi det end isse saa klarligen af det Ord , stm isse kan feile , at der er Ingen , som er retfcerdig , end Me Een , sta maatte vi dog stjonne , maatte vi dog , » aar vi isse vilde lyve mod <b>Sandheden</b> , demme om Pharisceenn med det ukjcerlige <b>Hjerte</b> , at han Me var en retfcerdig Mand . Hvad er det fremdeles , som gjor ham sta glad i sig stlv ? Han er Me en Tyv eller Rover eller Vellystig eller som denne Tolver ! Visselig , han havde isse , han vilde Me have nogen Tanke om , at Gud ster til Hjertet , at Gud begjcerer vort ganste <b>Hjerte</b> , vort ganste Sind , vor ganste Sjcel . Pharisceeren var reen i det Ivre ; dermed flog han sig til Ro , deri havde han Nok ; men dermed bedrog han sig <b>selv</b> , bedrog sig <b>selv</b> paa den sorgeligste Maade . Under det ydre Livs Reenhed kan dog Hjertet vcere et Syndens Barn , fanget i Morkets Lcenker ; ja , er den ydre Reenhed kun en Frugt af Tragten efter at vinde Menneskers 3 Ere , en Strceben efter at kunne sige om sig <b>selv</b> , at man er bedre end sine Brodre , da er den ydre 3Erbcrrhed for Hans <b>Die</b> , som gransker Hjerter og prover Nyrer , Syndens Vidnesbyrd og maa hoste Syndens Len . Havde Pharisceeren stuet ind i sit eget <b>Hjerte</b> , sta havde han sundet Nok til at blues , til at forfcerdcs over ; allerede hans <b>Hjertes</b> Villighet ) til at domme Brodrene havde overbeviitt ham , at han i Guds <b>Oine</b> var en Manddraber , og han havde vel vogtet sig for at anstille en Sammenligning mellem sig stlv og de Andre ; han havde Intet havt at rost sig af , han havde bekjendt sig kun en Synder at vare og tasset , takket ganske anderledes , end han gjorde , tasset , fordi Gud naadigcn endnu Me havde ladet ham doe i sine Synder . Og hvilke ere de gode Gjerninger , han opregner for sin Herre og Dommer , sig til Hceder og 3 En ? Han faster to Gange om Ugen og giver Tiendc af alt det , han eier ! Det arme , forblindede Menneskehjerte ! At faste og give Tende , mener han , er Noget , som ret maa vare et synderligt Beviis paa hans Gudfrygtighed og Fromhet», som om dette Me var Noget , der <b>selv</b> for Kjod og Blod er meget let , nåar man derved kan vinde Nogel , end sige , nåar man indbilder sig derved at kunne kjobe Frihed fra Dommen og det evige Livs Glade . Var det evige Liv at faae ved at faste og bortgive sin Eiendom , vare der vel Faa , stm isse gjerne bortgav Alt , hvad de eiede , gjerne fastede sig ihjel . Hvad siulde Me et Menneske ville give for sin Sjcel ? Ja , i Grunden var vel Enhver villig til for den Priis at ofre Alt , kun isse sig <b>selv</b> , Me sin egen Dygtighed , sin egen Fortjeneste . Alt vil Mennesket, kun isse modtage Naaden i Christo for Intet , og det igjen , fordi

<h4>2367</h4>Det er et ganste andet Billede , Medforloste , der <b>moder</b> os , nåar vi vende vort <b>Die</b> til Tolderen i Dagens Evangelium . Vi vide , at Tolderne vare stare foragtede mellem loderne , og det ei aleneste fordi de stode i de hedenske af loderne paa een Gang foragtede og hadede Romeres Tjeneste , men ogsta fordi de i Almindelighed vare bedragerske Mennesker , der for egen Vindings Skyld siammelig besvege baade dem , for hvem , og dem , af hvem de indlrcevede Skat og Told . Ogsta den Tolder , paa hvem vort <b>Die</b> nu hviler , havde maastee varet ligedan ; maastee var det let for Pharisceeren og for alle de Andre at overbevist ham om , at han mangen Gang havde aabenbart overtraadt Guds Bud , klarlige » var Dommen hjemfalden . Og . dog , sijont Ingen af os vil ncegte , at Synd er Synd , at stiv det naturlige Menneske baade bor og kan bekjcempe Syndens Udbrud i stige Gjerninger , og den nagler det mindst . som , <b>selv</b> falden , fornemmer Dommen i sit Indre , desuciglet hviler vort <b>Oie</b> mcd <b>Kjcerlighed</b> paa Tolderen fremfor paa Pharisceeren . Pharisceeren staaer der sta , let og glad i Hjertet , med sreidigt Mod og havet <b>Blik</b> ; Tolderen staaer der med nedslagne Dine og en Smerte i Hjertet , sterre og dybere , end at du fatter den , dersom du Me stlv har fornummet den samme Smerte . Og dog , sijsnt Tolderen sta klarlig staaer for os som en Bcnonr , en Smertens Sen og Jammerens Barn , dog maatte vi ti Gange hellere vcelge hans Smerte end Pharisceenns Glade , dog maatte vi vel onste , at vi vare i hans Sted , saadan stm han stod der , beiet for sin Herre og Gud med <b>Sandhcd</b> i Hjertet og oprigtig Von paa sin Tunge . Herrens Aand havde faaet Bugt med

<h4>2738</h4>Hans Salig er Glcede . Er der noget Ord , Verden tager forfengcligt, sta er det visselig dette . Hvad er Glede ? v det er isse Overflodighed af Nytelser , itte Tilfredsstillelse af en eller anden Lyst — nei den rene , den sande Glede er dette , at Hjertet varmt fyldes af og foler det sande Gode . Moderen gleder sig <b>selv</b> under Fodstlssmerter , fordi hun foler den Nyfodtes <b>Hjerte</b> at banke under sit ; staledcs udbreder hiin Salighcd under Smerten sin stirde Glede i vor Barm ; den aabncr nemlig for vort indre <b>Oic</b> Skutt ind over de Egne , som for van stjultc for os . Guds Forsyns Veie , altid en Hemmelighed for os , ligge itte lengere i Morke , men vi stue dem i Aandens Lys og glede os i den salige Folelse af vor Fvrlosning ; hvad der for var det naturlige Menneske en Daarlighed , bliver os til Viisdom , alt eftersom Aanden giver os at forstaae det , og gledes vi , nåar vort legemlige <b>Blik</b> efter et langvarig! Morke paa engang sik ste ud over en herlig Egn , hvormeget mere og sanden maa da ittc dcn Glede ven , stm tilbydes os , nåar vi stue <b>Sandhedens</b> Vidder nabne sig for os . Hvad Glede siulde vel kunne nenne sig den , at den Tunge , stm str knn bandede , nu lerte at velsigne, at det Sind , der fer var tilboieligt til Vrede og Mjerlighed , nu kan beseire sig <b>selv</b> og kan endog vist <b>Kjerlighed</b> og glede sig deri ? Og dctte er Hans Gjerning med os , at vor solvendte Natur bliver beseiret og den ondc Tilboelighed kuet , og vi altsta nu stae Lysten til at gjore det Gode for Guds Skyld , for at fremme Hans Villie , efter hvilken vvr bliver dannet . Da er Hjertet opfyldt , nåar Gud fylder del , da banker det varmt , nåar Hans <b>Kjerlighed</b> er Gloden , stm blender derinde , da ere vi salige og have i dette Salig vor Glede .

<h4>2795</h4>din Neste , kun en Pligt imod ham — den at kunne vinde hans Sjel for <b>Sandheden</b> . Og hvad enten han er med eller mod dig i timelige Ting , sta vil kun det Forhold , der bestemmes af denne holere , aandelig Kjerligheds Trang , vere det , som kan danne et Gjenskin af den himmelske Faders . Og ingen Sjel er da mere trengende til <b>Kjerlighcd</b> end den , som stlv er fremmed for den , ligesom det netop var Tollen og Syirdcrc Jesus var kommen at frelse ; jo mere Ukjerlighed , Had vg Forfolgelse , di » Neste viser dig , des mere trenger hans Sjel til <b>Kjerlighed</b> , des mcrc er det din Pligt ved opofrende , overdennde , <b>sand</b> christelig <b>Kjerlighed</b> at omfavne ham <b>selv</b> i hans Vrede . Og forstacer du ilke , hvorledes dette kan stee , sta st ind i det Forhold , bvor der endnn er mcest Num for den sande , frelsende <b>Kjerlighcd</b> — st , hvorledes Moderen ittc blot villig hengiuer sit Liv , men alle sit Livs Timer , al sin Ro , sin Glede , sin Fred for at redde Barnet fra timelig eller evig Dod . Var der en saadan Modcrsjel i vort Bryst ! det er dog steet og tunte endnu stee , at du kuude vinde dcn hidtil Uvilliges <b>Hjerte</b> ; tenk dig da , nåar du paa hiin Dag treder frem for din Konges Throne vg den Broder , du vandt , kommer med dig og du kunde sige : st , Fader , denne Sjel var vildfarende , og jeg var Midlet , Du brugte til at faae ham til Dig og Din Frelse , han er Din — o stal <b>Qvinde</b> » frelses formedelst Barnefodscl , sandelig det vil ven godt , at den til Ansvar krevende Rost er forandret til en talncmmelig lubelsang . „ Forlad dine Skyldnere " , forlad stm du <b>sclv</b> vil vere forladt ; thi sandelig , uden <b>Kjerlighed</b> er tet umuligt at behage Gud , og enhver ukjerlig Folelse vil ligge trykkende paa din Sjel indtil den , hvis den Me bliver afkastct, engang vil trytte dig ned der , hvor der er Graad og Tenders Gnidsel , hvor de Ukjerlige boc .

<h4>2973</h4>kommer og gaacr og dog aldrig metter den higende Sjel ; Jorden laa under hans Fedder , og dens Kaar vare itte i hans Tanker . Hvad var det , som fyldte hans Sjel ? O , det var den Fred , som Verden itte giver , og Verdcn Me tager , det var den fuldere Ahnelse af det evige Livs Herlighed , som nu aabenbandes for ham , det var Troen , som var bleven en Bestnelse . Hvor mnatte ittc Mindet om denne Time folge ham styrkende i Livet og vidne for ham , „ at han Me , nåar han forkyndte den Herre , paa hvem han troede , fulgte klegtige Fabler ; thi han havde <b>selv</b> vent Oienvidne til Hnns Majestet " . — „ Her er godt at vere " , disse Ord kunne vel ogsaa hores hcrnede , nåar et <b>Hjertes</b> Dnste er opfyldt , og det for en Stund er bleven stille derinde , nåar Dagen gaacr let , og Livets milde Glede meder Mennesket . Men med storre <b>Sandhed</b> siges det vog kun da , nåar det Evige treder dig ner , nåar Guds Tanker fulde Sjelen , og det Forkrenkelige er flyglet fra dit <b>Oie</b> . Og aldrig sinder dog Hjertet sin Fred , forend det har sundet Ham , „ ved hvem og til hvem vi Alle ere stable " , strend Guds Tempel er reist i Hjertet , og Fadernavnet er lagt paa vor Tunge , og det evige Livs Haab byer i Sjelen . Og der er et Ord , Brodre ! som ofte lyder fra den hellige Dng , — og lad det atter hores under det stygtende Aar — Doden trengte gjennem til Alle , fordi de syndede Alle . Der er en Skilsmisse mellem Gnd og det naturlige Menneske , og visselig vare vi Alle fvrmedelst Dodens Frygt skyldige til Treldom al vor Tid . Ittc al din Streben efter Rctferdighed , itte alle dine egne Ofre kunne bringe dig til Gud , stalidt som et „ du stal " kan fremkalde <b>Kjerligheden</b> , kan fornye dig til Guds Billede igjen ; derfor maatte Gud — thi ene Han kunde det — stlv grunde vor Frelst ; derfor „ kommer Ingen til Faderen uden gjennem Sonnen . og Ingcn kjender Faderen , uden Sonnen og den , stm Han vil dct aabenbare " . Har <b>Hjcrtet</b> da fundct sin Fred , kan du med Troens Tillid hvile i Gud , rolig hore alle Dodens Stemmer og med Frimodighet » stue ind i den Herlighcd , hvor nllc Tunger stulle juble „ her er godt nt vere , " da var det , fordi du erkjendte din Guds Stemme ogsta gjennem Lovens Dom , da var det , fordi du ei vilde hvile alene i dunkle Prvphetier eller i dit eget Haab paa Guds Barmhjertighed , der , hvis det er udsprungen af Kjod , ogsaa kun er Kjed , men fulgt af Moses og Elias sogtc til Ham , der staaer med Guddomsherlighedeu i sit Aasyn og Forsoningens Kalk i sin Haand , da var det , fordi du i Troen greb Ordet „ Gud var i Christo og forligte Verden med sig <b>selv</b> " . O ! maatte den gode , den Hellig-Aand , som udgaaer fra Faderen og Sonnen , som virker i de Dobles Sjele og kalder til Guds Rige , maatte Han fore os

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016050948039'target=_>Walther, C.F.W., 1878, Amerikansk-luthersk Evangelie-Postille</a><h4>61</h4>„ Lcere Frihed " , ikke fordi han tror , at enhver Vildfarelse mod Guds Ord i og for sig fordommer , men fordi han anfer det for Kirkens , og isoer dens Tjeneres hellige Pligt „ at stride for den Tro , som engang er overantvordet de Hellige " ( Ind . V . 3 ) , og desuden tror , at enhver Vildfarelse , isoer naar den vinder Fodfceste i Modscrtning til <b>Sandheden</b> , baade er farlig for den enkelte Sjæl og skadelig for Kirken , idet den svcekker Opfyldelsen af det Ord : „ Hold fast ved den rette Form af det sunde Ord " . Walther indlægger alvorlig Protest imod Nutidens saakaldte „ Fremskridt " i Theologien og dens foregivne „ Laereudviklinger " , ikke fordi han tror , at Skriftens Rigdomme ved de forrige Aarhundreders Arbeide ere udtomte , eller at ogsaa Formen i den gamle Dogmatik var et Stykke af Rettroenheden , men fordi han finder , at Nutidens Theologi endnu ikke for Alvor er vendt tilbage til den gamle Grundvold , men blot soger at skjule sit umiskendelige Frafald fra den gamle lutherske og overhovedet kristelige Troes Grundvold og falbyder gamle forslidte Kjcetterier under Navn af nye beundringsværdige Opdagelser i Theologien . Walther tror , at den gamle Tro vistnok taaler en uendelig Udfoldelfe , og at den hellige Skrift er en uudtommelig Kilde til guddommelig <b>Sandhed</b> og et Rustkammer , som bestandig byder os nye Vaaben mod nye Vildfarelfer eller mod gamle Vildfarelser i ny Skikkelse , men han forkaster alligevel den Mening som . en Vildfarelse , at enten vor egen Tid eller nogen anden Tid engang skulde kunne opdage eller opfinde en Troesartikel , som indtil da skulde have været ukjendt for den rettroende Kirke . Hvor meget han end holder af de Vidnesbyrd om <b>Sandheden</b> , som han finder i Luthers Skrifter og ide kirkelige Bekjendelser , dernoest ogsaa i vore anerkjendt rettroende Lærefoedres Skrifter fra det 16 de og 17 de Aarhundrede , saa holder han dog med al Kraft fast ved det lutherske Skriftprincip , at alene de kanoniske Boger ere den guddommelige <b>Sandheds</b> Kilde og en afgjorende Dommer i alle Lærestridigheder . Walther afskyr enhver saakaldt „ Videnskab " , som opkaster sig som Mester over den guddommelige Aabenbaring og dens Hemmeligheder , men han holder ligefuldt fast ved den Erkjendelse , at en sund Videnstab som en » aneilla tneoloFiss « , d . e . en <b>Tjenerinde</b> for Theologien , er af überegnelig Værdi for Kirken til den rene Læres Bevarelse . Han anerkjender ingen rettroende Lærers Mening om en Ting , der ikke er Guds Ord og ikke kan afgjores deraf , for at være nogen bindende Afgjorelse af Sagen , ' men anser en saadan Meningsytring for i hvert Fald „ problematisk " ; men paa den anden Side forkaster han paa det Bestemteste det Princip , at alle de Loerdomme , som ikke Kirken har afsluttet eller afgjort i sine Symboler , sinlle være „ aabne Sporgsmaal " , hvor det blot gjælder en menneskelig „ Mening " , og hvor selvfslgelig en „ berettiget Lærefrihed " maa raade . Walther er en afgjort Fiende af pietistisk LovtrceldoM — thi baade Skriften og hans egen bitre Erfaring har beloert ham om , hvor farlig og fordærvelig denne „ Retning " er — men han er derfor ingen Ven af en saakaldt „ dod Orthodoxisme " . Han holder fast ved , at den sande „ levende Kristendom " hverken bestaar i en Folelsestro , som man <b>selv</b> har arbeidet og kjcempet sig op til , eller i en Tro , som formedelst naturlige Evner er bleven til fom en menneskelig loererigtig Overbevisning om kristelige <b>Sandheder</b> , men at den bestaar alene i en Tro , som er bleven virket af den Hellig Aand i den nye Fsdsel , ved Evangeliet , — i et <b>Hjerte</b> , som ved Loven er bleven forstroekket og frygter for Guds Dom , — en Tro , som griber Kristum i de evangeliske Forjættelser , forer den Hellig Aand med sig , omskaber Mennesket , helliggjor og fornyer det og gjor ham rig paa <b>Kjærlighed</b> til Gud og Noesten og villig til enhver god Gjerning . Hovedlærdommen er for Walther , som det for en god luthersk d . e . bibeltro Kristen og Prcest altid bor være det , Læren om Retfcerdiggjorelsen af Troen alene uden al vor egen Bedens , Kjcempens , Arbeides osv. Fortjeneste . Kristus bliver tidlig og sent af ham fremholdt som den Troendes eneste Trost og Haab , fordi han „ er hengiven for vore Synder og opreist til vor Retfoerdiggjorelse " . Deraf folger ogsaa

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013072208224'target=_>, 1880, Vidnesbyrd fra den norske Kirke</a><h4>239</h4>Kristi gode Evangelium ikke er blevet troet ! Kristi Rost er Freds Rost , Kristi Sind er Selvfornegtelsens Aand , Kristi Gave er <b>Kjcerlighedens</b> Hjertelag , og rundt i Kristenheden hores i de storre Samfund Krig og Rygte om Krig , i de mindre Samfund Uenighed og Splid , at den Ene misunder den Anden , at den Ene forfordeler den Anden , at Fordragelighed kun kan bevares under Skilsmisse . Kristi Sind vender <b>Blikket</b> indad og boier Hjertet til Idmyghed , og i hvad der kaldes kristne Samfund , hsres Dommen over Andre at hceve sin Rost , medens Sporene af , at den Samme dommer sig <b>selv</b> , ere skrevne i Flyvesand ; Hovmodens Pande lofter sig frcekt , og saa Mangen begjcerer aabenbarlig at vcere den Forste for at herske efter sit Hoved . Kristi Sind gjor os til Udlændinge i Verden og stab er Lcengfelen , den stille , men daadkraftige Lcengfel efter Hjemmet heroventil, og se , midt i Kristenheden fcerdes Folkene , som om her var deres blivende Sted ; her er deres Liggendefæ , her ville ogfaa deres Hjerter vcere ; i det , fom er idag og imorgen kastes i Ovnen , deri soger man sin Gloede og sit Liv ; dets Fylde er Herlighed , dets Savn er Kval . Kristi Sind er Alvorets Nand ; Kristi Tro staber Mcend , som vide , hvad de ville i Herren , <b>Kvinder</b> , som vide med Hjertet , hvad de elste i Herren , og midt i Kristenheden tumle sig de Mange , hvis Liv er mcerket med den flagrende Forfængelighed , hvorpaa de have kastet , hvad af en stakkels <b>Kjcerlighed</b> det fattige <b>Hjerte</b> eier , de Mange , hvis Idrcet er den tomme Forfængelighed , hvad enten den er Arbeidet kun for det , som horer denne Verden til , eller Glceden kun i det , som er af denne Verden , ak , altfor ofte idel tom Forfængelighed, <b>selv</b> om den laaner en Form af et aandeligt Vcefen , der altfor fnart aabenbarer sin Formhulhed ved sin Halvhed og sin Flygtighed . Kristus Jesus gjor Menneskenes Born velbehagelige for Ham , hvem Ingen stal fe uden Hellighed , renfer deres Hjerter og bliver <b>selv</b> deres Aands Liv og Tragten ; men i Kristenheden hores ikke alene om enkelte Misgjerningsmcend , men Manges daglige Liv gaar gjennem Overtrædelser , om ikke af Menneskers , saa dog af Guds Lov . De tjene uden Frygt sine egne Begjceringer og have Mod og <b>Hjerte</b> til at bruge og forfvare , hvad de dog felv vide at vcere mod Aanden . Sandelig , det er mere end forunderligt , at Guds store og gode Gjerning og Guds Vudstab om sin Gjerning har indtil denne Stund udrettet saa Lidet i Verden , faa Lidet mellem os , saa lidet er blevet modt og brugt med Tak og Tro . Det er mere end forunderligt , at faa er steet , og at Gud dog bliver ved med at forkynde sit Vudstab , at tilbyde os sin Gjerning ; vor forunderlige Gud kalder sig jo ogfaa <b>selv</b> Taalmodighedens Gud .

<h4>241</h4>Tet er ikke glade lulebilleder , som her ere dragne forbi vort Vie , men , Medforloste , medens jeg maa byde Menigheden , hvad jeg har , faa I <b>selv</b> domme , om disse Billeder dog ikte ere af <b>Sandheden</b> . < „ Men Wre vcere Gud i det Hoie ! dette er dog heller ikte den hele <b>Sandhed</b> . Det er til vor og vor Slcegts Vancere foranderligt, at Julebudskabet er saa lidet og saa lidet fuldt inderligt troet ; men det er til Guds Naades Pris og 3 Ere dog ogsaa <b>sandt</b> , at det er forunderligt , at det dog ogsaa er troet , er troet saaledes , som det aabenbarlig er det . Med Viet fcrstet kun paa Guds Kærligheds Dyb , den aabenbarede Frelsens Hemmelighed og vor Lod uden saadan <b>Kjcrrlighed</b> og uden saadan Frelse , bliver det saa saare foranderligt , at de fortabte Mennesteborn ikke stromme til og kjobe foruden Penge , ikte styrte fig med idel Tak og Jubel ind i Guds aabnede Faderfavn , de famme Mennester , fom ellers faa villigt flokke sig om den stakkede Glcede , faa let , ja letfærdigt stjcenke Enhver fm Tro fom lover Guld for godt Kjob . Fceste vi derimod vort <b>Blik</b> paa ' hvad <b>Sandheden</b> lcerer om det naturlige Menneskehjertes Tilstand og Forhold til <b>Sandheden</b> og til Naaden , ak , da bliver det forunderligt , at Guds <b>Kjcerlighed</b> , faa rig og faa dyb , fom den er , at Frelfens Hemmelighed , hvor aabenbaret den end er bleven , nogenfinde er troet af Nogen . Det vide vi jo Alle , at hvad man ikke vil fe det kan man ei heller fe , og det er Me lysteligt at se sin Sja-ls Skade at sijonne for vist , at al vor Retfærdighed er et befmittet Klcedebon Lidt maaste , maafke Meget ville vi indromme at vidne mod os ; men at Alt gjor det , at vor bedste Gjerning fortjener Vredens Dom , det er for haardt at gaa paa ; at den , fom frit kan fe hver Mand paa Jorden i Viet , stal med stor Skam og Blufel maatte flåa Viet ned for den levende Gud , det er dog for strengt for vor Sjcels Tro , for fvcert for vort <b>Hjertes</b> Bekjendelse . Vi kunne indromme , at denne Sag er af Vigtighed , at den fortjener vor Opmærksomhed, men at vedgaa for vort eget <b>Hjerte</b> , at denne er for Enhver af os den allervigtigste , fom krcever vor Opmærksomhed meget mere , end Sporgsmaalet om , hvormed vi stulle fode og klcrde os end sige , hvormed vi skulle more os , at det gjcelder Liv og Dod at ' komme til <b>Sandhedens</b> Erkjendelse , og at dette er <b>Sandheden</b> , at , dersom vi <b>selv</b> skulle fore vor Sag , ere vi hjemfaldne den evige Dod , ak det have vi meget let for at indbilde vore Hjerter at vcere idetmindste en svcermerisk Overdrivelse . Mode vi en alvorlig Mand med en saadan Bekjendelse , have vi strax Dommen paa det Rene ym en mork Livsbetragtning , maaste tilmed om et bittert Sind , for

<h4>538</h4>hvide rene Klædning , du da sit , saa kom som en angrende Son og Datter igjen tilbage til den Fader , som dn har bedrsvet , og du vil igjen faa det „ bedste " Klcedebon som Barn i Huset . Han vil rore ved dig , naar Han lader Evangeliet ved den Helligaands Kraft slaa an i dit <b>Hjerte</b> . Lad saa endvidere Jesus rsre ved dig , idet Han gjennem sine Tjenere som Svar paa din Bodsbon forsikkrer dig om dine Synders naadige Forladelse , lad Ham rsre ved dig , idet Han ved Nadverbordet kommer usynlig og forklaret til dig og meddeler dig sit himmelsk forklarede Legeme og Blod . Ja . i <b>Sandhed</b> du stal faa erfare Hans Nwrvaerelse , Hans <b>Kjoerlighed</b> og Kraft , og du stal faa hore Hans Tale til dig gjennem alle Hans Naades Midler : Jeg vil , bliv reen . Naar saa Frygten for din egen Uværdighed og Tilliden til Hans <b>Kjcerlighed</b> og Magt kæmper mod hinanden i dit <b>Hjerte</b> , saa forehold dig Hsvedsmandens Exempel . „ Herre , jeg er ikke voerd , at du trceder ind under mit Tag . " Nei , heller ikke du og jeg er det voerd , og endnu mindre v « rrd , at Han troeder os endnu ncermere , traeder ind i vort arme <b>Hjerte</b> . Og dog vil Han nedlade sig faa dybt , stige ned til os , paa det inderligste forene sig med os , komme til os og bo i vort <b>Hjerte</b> , Han i os og vi i Ham . Det er en forunderlig dyb , nedladende Kj « erlighed . Men saa er det l Virkeligheden . Han elsker os langt inderligere , alvorligere og stcerkere end vi formåa at tcenke og at tro . Folelsen af vor Uvaerdighed maa forblive der , Ydmyghed , dyb Ydmyghed er et blivende , et nodvendigt Kjendetegn paa Guds Bsrn , men den bor aldrig fordunkle Synet paa Jesu CH risti Magt og <b>Kjcerlighed</b> og borttage Tilliden til Jesu Naadesord . Tvertimod jo mindre og uværdigere du bliver i dine egne <b>Oine</b> , med desto fastere Tillid maa du klynge dig til Jesus og holde dig urokkeligt og uden at flippe Taget fast ved Hans blotte Ord og Vidnesbyrd i Evangeliet . Som Ydmyghed og Tro vare saa stjont forenede hos Hsvedsmanden , faa vcere de det ogfaa hos os . Lader os kun se paa os <b>selv</b> , paa vore Synder , vor Elendighed og Afmagt , til vor stadige Ydmygelse , men lad os ogsaa paa samme Tid se tillidsfuldt paa den forbarmende , kjaerlighedsrige Frelser paa Hans korsfoeftede og gjenopftandne , forherligede , evigt levende Person , paa Hans Ord og Hans Gjerning , og lad os have Ham levende , fom Han er , midt iblandt os i Ord og sakrament. Lad os hore Hans levende Rost i Hans levende Ord . Det er den evige <b>Sandheds</b> , den evige Almagtens Ord , og vi stulle erfare , hvad Hsvedsmanden erfarede , at Hans Ord er almægtigt og skabende . O maatte vi , saamange som gjerne vil hore Jesus til ,

<h4>843</h4>Faren renset i Blodet fra al Synd og Uretfærdighed og prydet med sin Blodbrudgoms stjsnne Smykker , det bedste Klcedebon , Bryllupskjortelen, Trolovelsens Ring og Sko paa Jodderne , disse ombundne med Færdighed i Fredens Evangelium , den sagtmodige og stille Aands uforkrænkelige Vcesen , som er meget kosteligt for Gud , i helligt Levnet og Gudfrygtigheds øvelser , en Vandel i Jesu Fodspor . Mod et saadant <b>Hjerte</b> , ifsrt Guds fulde Rustning , formaar Satan Intet . Med den Stcerkere vover han sig ikke mere i Kamp . Men opgive Hjertet vil han endnu ikke . Og saa gjcelder det Forstærkning , og han gaar hen og tager syv andre Aander med sig , som ere vcerre , end han <b>selv</b> . Ligesom Herren udfolder sin Fylde af gode Gaver ide syv Guds Aander ( Aab . 3 , 1 ) , til Forsoningens Tilegnelse , saaledes har og Djcevelen sin Fylde af onde Krcefter i de syv onde Aander , som ere vcrrre end den urene Aand , vcerre i indre Ondskab , frygteligere i Magt I den forfærdelige Ondskabens aandelige Hcer under Himmelen er der Fyrstendommer og Magter , altsaa Befalende og Lydende < Efes 6 12 ) I Kampen mod Guds Rige fremtræder Satans Rige som en ' ordnet Hcer , hvori Enhver er paa sin Post . Frygtelige <b>Sandhed</b> ! Sandelig , Djcevelen jager man ikke vcrk ved at negte hans Tilværelse . Negtelsen af hans Tilværelse er et af de alvorligste Vidnesbyrd om hans frygtelige Vcelde over Vantroens Born og . hvor den griber om fig , om Frafaldets Storhed og om , at de sidste Tider ere komne . Han kommer altsaa igjen . Som med den kjcere Frelser gaar det ogsaa med Hans Brodre hernede , kun for en Tid viger Satan for den Stcerkere . Og naar han kommer igjen med sin Forstærkning, og de faa , som ofte sker , komme ind , bo de der ; og det Sidste bliver vcerre med det Menneske end det Forste . Ja , naar . . de som have undflyet Verdens Besmittelser ved den Herres og Frelsers Jesu Kristi Erkjendelse , igjen lade sig indvikle deri og blive overvundne , da er det Sidste med dem blevet vcerre end det Forste . Men hvorledes er det gaaet til , at den urene Aand forste Gang afvistes , men nu kommer ind . Ligger Grunden i den storre Magt og List , hvormed han nu angriber ? Dette er visselig ikke uden sin Betydning ; men Grunden ligger dog vcesentlig paa den frelste Sjcels Side . Den er ikke blevet nvidende om den urene Aands forste Angreb . Meget mere den lykkelige Sjcel har ikke blot meerket det , men salig i sin Frelsers Magt og <b>Kjcerlighed</b> har den med <b>Blikket</b> paa Ham og Hans troocrrdige Ord vcrret stcerk og i sin Herres Rustning sejret . Og faar Herren bevaret en Sjcel i dette Sind og i dette Forhold , kunne alle Ondskabens Hcere , fom Helvedes Porte udfende , Intet udrette mod et Guds Barn . Maatte dog alle oprigtige

<h4>2186</h4>Barnet , der har klatret op paa et utrygt Sted , at kaste sig i Armene ylla Faderen , der bekymret opmuntrer det til at kaste sig i hans stcerke , aabneFavn ; men som Barnet dog tilsidst vover det , tilskyndet af Faderens <b>kjcerligt</b> opmuntrende Stemme , saaledes hjcelper Aanden dog til at fortroste dig til din Frelsers Ord , til at forlade den Grund , hvorpaa du hidtil har sogt Fodfceste , til at opgive dit Eget og kaste dig i hans Favn i Tro paa hans Naade og Sanddruhed , og du bliver ikke skuffet . Med glad <b>Hjerte</b> faar du erfare , hvad din Styrke eller Fornuft ikke var mcegtig til at klare for dig , at han er Veien , er <b>Sandheden</b> og Livet , og at han er det , just fordi han er Gud og Menneske i een Person , som , fordi han er Davids Son og fandt Menneske , har kunnet lide og ds for dig og alle Mennesker, som , fordi han er Guds Son , <b>selv</b> Gud , har en evig Kraft til at forfone den Gud , mod hvem Synden er Opror . Du erfarer det , idet du faar Hvile for din Sjcel , faar Syndens Byrde vceltet bort fra dit <b>Hjerte</b> , faar en let og glad Samvittighed . Da kan du aande frit som den , der er frikjendt og stuppen ud af et Fcengsel , tan lsfte dit <b>Oie</b> og din Tanke freidigt til din og lefu Kristi Fader i Himmelen, fordi dn nu ved og tror , at han ikke mere er vred paa dig , men er din forsonede , gode og naadige Fader , og du hans kjcere Barn for Sonnen , den elskeliges Skyld . Og dette Aandens Vidnesbyrd gjennem Ord og Sakrament , det er dig hoiere end alle din naturlige Forstands Modbeviser , denne dit <b>Hjertes</b> Erfarings glade Overbevisning er dig stcerkere og mere vcrrd end dit Kjods vantro Indfigelfer , er dig det fuldgyldige Bevis for den gamle Kristentroes <b>Sandhed</b> . Hvad Forstanden ikke kan faa til at hcenge sammen : at den evige Gud engang i Tiden blev Menneske , at en Andens , Jesu Kristi , Gjerning og Lidelse er en Forsoning for dine Synder , de store med de smaa , det fatter du i Troen , og paa Sporgsmaalet : „ Hvad tykkes dig om Kristus ? hvis Son er han ? " kan du svare af Hjertet og i <b>Sandhed</b> : Jeg tror , at han er Guds Son og Davids Son , at han er fand Gud , fodt af Faderen fra Evighed ; jeg tror ogsaa , at han er <b>sandt</b> Menneske , fodt af Jomfru Maria , at han er min Herre , fom har igjenlost mig , der var fortabt og fordomt til den evige Dod , og fom har friet mig fra Synd og fra Dod og fra Satans Rige , hverken med Guld eller Solv , men med sit hellige og dyrebare Blod og med sin übillige og uskyldige Dod . Og dette har du den Hellig Aand at takke for , fom ydmygede og boiede dig ved Loven for saa at kunne opreise og troste dig ved Evangeliet . I den samme Aands Kraft vil du sige med Fodfceste paa Klippen Kristus :

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015110407545'target=_>Walther, C.F.W., 1878, Evangelie-Postille</a><h4>26</h4>„ Lcere Frihed " , — ikke fordi han tror , at enhver Vildfarelse mod- Guds Ord i og for sig fordommer , men fordi lian anser det for Kirkens , og iscer dens Tjeneres hellige Pligt „ at stride for den Tro , som engang er overantvordet de Hellige " ( Iud . V . 3 ) , og desuden tror , at enhver Vildfarelse , iscer nåar den vinder Fodfceste i Modsætning til <b>Sandheden</b> , baade er farlig for den enkelte Sjæl og skadelig for Kirken , idet den svækter Opfyldelsen af det Ord : „ Hold fast ved den rette Form af det sunde Ord " . Walther indlcegger alvorlig Protest imod Nutidens saakaldte „ Fremskridt " i Theologien og dens foregivne „ Lcereudviklingcr " , ikte fordi han tror , at Skriftens Rigdomme ved de forrige Aarhundredcrs Arbeide ere udtomte , eller at ogsaa Formen i den gamle Dogmatik var et Stylte nf Rettroenheden , men fordi han finder , at Nutidens Theologi endnu ikte for Alvor er vendt tilbage til den gamle Grundvold , men blot soger at skjule sit nmistjeudclige Frafald fra den gamle lutherske og overhovedet kristelige Troes Grundvold og falbyder gamle forslidte Kjætterier under Navn af nye beundringsvcerdige Opdagelser i Theologien . Walther tror , at den gamle Tro vistnok taaler en uendelig Udfoldelse , og at den hellige Skrift er en uudtommelig Kilde til guddommelig <b>Sandhed</b> og et Rustkammer , som bestandig byder os nye Vaaben mod nye Vildfarelser eller mod gamle Vildfarelser i ny Skikkelse , men han forkaster alligevel den Mening som en Vildfarelse , at enten vor egen Tid eller nogen anden Tid engang skulde kuune opdage eller opfinde en Troesartikel , som indtil da skulde have været ukjendt for den rettroende Kirke . Hvor meget han end holder af de Vidnesbyrd om <b>Sandheden</b> , som han sinder i Luthers Skrifter og i de kirkelige Bekjendelser , derncest ogsaa i vore cmertjendt rettroende Lcerefcedres Skrifter fra det l6de og 17 de Aarhundrede , saa holder han dog med al Kraft fast ved det lutherske Striftftrincip , at alene de kanoniske Boger ere den guddommelige <b>Sandheds</b> Kilde og en afgjorende Dommer i alle Lærestridigheder . Walther afskyr enhver saakaldt „ Videnstab " , som opkaster sig som Mester over den guddommelige Aabenbaring og dens Hemmeligheder , men han holder ligefuldt fast ved den Erkjendelse , at en sund Videnstab som en » anoilla tnsoloFiZL- , d . e . en <b>Tjenerinde</b> for Theologien , er af llberegnelig Vcerdi for Kirken til den rene Lcercs Bevarelse . Han anerkjender ingen rettroende Lcerers Mening om en Ting , der ikte er « åbenbaret i Guds Ord og ikke kan afgjores deraf , for at være nogen bindende Afgjorelse af Sagen , men anser en saadan Meningsytring for i hvert Fald „ Problematisk " ; men paa den anden Side forkaster han paa det Bestemteste det Princip , at alle de Lcerdomme , som ikke Kirken har afsluttet eller afgjort i sine Symboler , stulle være „ aabne Sftorgsmaal " , hvor det blot gjælder en menneskelig „ Mening " , og hvor sclvfolgelig en „ berettiget Lcerefrihed " maa raade . Walther er en afgjort Fiende af pietistisk Lovtrceldom — thi baade Skriften og hans egen bitre Erfaring har belcert ham om , hvor farlig og fordærvelig denne „ Retning " er — men han er derfor ingen Ven af en saakaldt „ dod Orthodoxisme " . Han holder fast ved , at den sande „ levende Kristendom " hverken bestaar i en Folelsestro , som man <b>selv</b> har arbeidet og kjæmftct sig op til , eller i en Tro , som formedelst naturlige Evner er bleven til fom en menneskelig lcererigtig Overbevisning om kristelige <b>Sandheder</b> , men at den bestaar alene i en Tro , som er bleven virket af den Hellig Acmd i den nye Fodsel , ved Evangeliet , — i et <b>Hjerte</b> , som ved Loven er bleven forstrcekket og frygter for Guds Dom , — en Tro , som griber Kristum i de evangeliske Forjættelser , forer den Hellig Aand med sig , omskaber Mennesket , helliggjor og fornyer det og gjor ham rig paa <b>Kjærlighed</b> til Gud og Ncestcn og villig til enhver god Gjerning . Hovedlcerdommen er for Walther , som det for en god luthersk d . e . bibel tro Kristen og Prcest altid bor være det , Lceren om Retfærdiggjorelsen af Troen alene nden al vor egen Bedens , Kjæmpcns , Arbeides osv. Fortjeneste . Kristus bliver tidlig og sent af ham fremholdt fom den Troendes eneste Trost og Haab , fordi han „ er hengiven for vore Synder og opreist til " or Retfærdiggjorelse " . Deraf folger ogsaa

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010021803052'target=_>Zschokke, Heinrich, 1857, Huusandagtsbog</a><h4>658</h4>- . Men bort fra disse Samfundets Skrekkebilleder ! Tanken om dem stal ikke besmitte min Andagtstime ! Hellere vil jeg kaste et <b>Blik</b> paa hine Ulykkelige , der uden egen Skyld maa savne Mgtestabets Gleder . — O , dadler ikke dem , men Samfnndet , der er Skyld i deres Ulykke ! Gders Datel er uretferdig ; ak , maastee har det kostet dem langvarig Kummer og dyb Smerte at stulle give Slip paa deres hoieste Onster og stjonneste Forhaabninger, fordi de ikke vilde gjore en brav <b>Hustru</b> til <b>Moder</b> for brodlose Vorn . Grusom er din Spot ! Det er en let Sag hos den raae Hob , der ikke untersoger <b>selv</b> men tager dine Vittigheter for <b>Sandhed</b> , at vekke Latter over Pebersvenden , som du kalder ham . Men vogt dig ; haus <b>Hjerte</b> er maastee mere Mre verdt end dit , og du forsynder dig maastee mod dets Allerhelligste; det fortjente maastee om Skaansel og venlig Trost , og du lader det vederfares haanlig Spot . Kjender du de Grunde , der kunne bevege den braveste Mand til at fordomme sig <b>selv</b> til at leve i ugift Stand . Maastee gjorde han det af Frygt for de vanskelige og stormende Tider og de stadige Omveltninger i det borgerlige Liv , under hvis Tryk saa mange hederlige Familier maae lide . Maastee traadte han tilbage af hoi og edel Besindighet » , advaret ved sin legemlige Skrobelighed . Maastee bragte han sin egen Lykke som et Offer for sine trengende Foreldres Fred eller for et andet hederligt Oiemed . Gller veed du , om ikke en tidligere ulykkelig <b>Kjerlighed</b> har belaget ham

<h4>1215</h4>Og du , hvis <b>Hjerte</b> med Omhed bandt dig til et andet , og som i dette sogte al din Lytte for Livet — hvi saa kummerfuld? Ak , dette <b>Hjerte</b> har stusset dig ! Hine Leber have kun fremhyklet en <b>Kjerlighed</b> , som du skjenkete med trofast Sjel ! Hine <b>Oine</b> have tilsmilet dig fulde af Falskhet » ! Din Trofasthet » er bleven gjengjeldt med Meened , din Omhed med stjendigt Forrederi! — Ulykkelige , du har tabt Meget ; denne Erfaring har folnet dit varme <b>Hjerte</b> maastee for stedse og berovet dig din Tro paa Menneskeheden . Denne Troloshed har maastee for stedse fyldt dig med Mistanke og Mistillit » til al menneskelig Dyd . Du har itte lengere nogen Ven , til hvem du kan hengive dig og betroe dig med fuld Tillid . Du er bleven eensom i Verden, og Venstabet var netop det , der havde storst Tillottelse for din blode Sjel . — Men tab itte Modet ! Strid ogsaa du den gode Strid , og fuldend Lobet som en <b>sand</b> Christen ! Intet bliver dig tro hernede nden din Gud ! Om og den ganske Verden stusser dig , Een er der , som aldrig gjor det ; det er din Gud , den Trofaste og Sanddrue , der <b>selv</b> har nedlagt de blide Folelser i dit <b>Hjerte</b> ! — Og , maa du vandre Livet igjennem uden nogen Ven , een Ven har du , som du altid beholder ; det er din evige Fader og Staber ! Og , nåar du maa miste det Kjereste og Dyrebareste , du vidste , saa lad dette Skyggespil , denne Afverling i det Foranderlige netop tjene til at styrke din Kraft og udvikle din Selvstendighet » og til at trenge dig hen mod det Blivende og Sande .

<h4>1313</h4>— Men , saasnart Petrus var kommen til sig <b>selv</b> efter cen forste Skrek , fandt han det hverken godt eller klogt , hvad han havde gjort . Han angrede sin Synd og gret » bitterlig . Han solte , at han itte havde gjengjeldt Frelseren hans uendelige <b>Kjerlighed</b> . Hvilken Fader vilde kunne fornegte sit Varn i Farens Stund ? Hvilken <b>Moder</b> vilde itte hellere vove sit Liv for sit Barn , end frasige sig det i Noden ? <b>Sand</b> <b>Kjerlighed</b> tenker kun paa den elskede Gjenstand og glemmer sig <b>selv</b> . Petrus solte , at han itte havde elsket Jesus med den samme <b>Kjerlighed</b> , hvormed denne havde elsket ham . Derfor gik han bort og udgod bittre Angerstaarer . — Og han folte , at hais havde handlet itte blot ukjerligt men ogsaa uretferdigt . Thi det kom jo her ikke an paa at frelse sit eget eller Christi Liv men paa at give <b>Sandheden</b> Wren , den <b>Sandhed</b> , som Jesus uforferdet og uden at endse sine Dommeres Vrede , havde vidnet . Det kom an paa , at der ligeoverfor de falske Vidner ogsaa optraadte sanddrne og sagde : Denne Jesus er Messias , Verdens Frelser ! Det var as Vigtighed for at styrke de Troendes Mod , at Jesu Disciple offentlig handlede i hans Aand . og at hans Evangelium viste sig verdigt til , at mau offrede Liv og Vlod derfor . Men Petrus havde itte Mod til at vere den Forste , der med sit Vlod beseglede sin Tro . Maatte det itte nedsette Christus <b>selv</b> i hans Fjenters <b>Oine</b> , at ingen af hans Disciple havde Mod til i Afgjorelsens Time at ertlere sig for at hore ham til ? Alt dette betenkte Petrus itte i siv forste Strek . Undseelsens og Angerens Taarer faldt for silde . Mett Petrus fornegtede Jesus kun i den forste Skrek ; hvor Mange ere der itte nu , som kalde sig hans Disciple , og som dog fornegte ham lige overfor Verden !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010092803030'target=_>Luther, Martin, 1855, christelig Postille</a><h4>353</h4>sigende : jeg har en Frelser , Himmeriges Nige er mit , ChristuS , Guds Son , er min ! Man forfola / mig kun for denne min Troes og Bekjenbelses Skyld , jeg vil dog ei blive utaalmodig eller harmfuld , og ingen Kummer og Lidelse , saa stor den end er , stal vcere mcegtig til at forbittre og fordarve mig den Lyst og Glade , jeg faaer af dette nyfodte Barn . Et saadant <b>Hjerte</b> onste de kjcere Engle alle Troende , at de maae have Glade i Sorg , og synge , naar Djavelen er vred , at de maae have Fred og et hoit Mod i Christo , ja formedelst Ham trodse al Ulykke og spotte Djavelen , naar han stader dem , sigende : Hor , Djcrvel , kan du ei gjore andet , end angribe mit Legem , Liv og Gods , saa kan du ligesaa godt lade vare , thi du kan ei skade mig : jeg har en mcegtig Frelser , Han vil med evig Fryd erstatte mig alt timeligt Tab . Et saadant <b>Hjerte</b> , der beholder sin Trost og Glade , om det end gaacr not saa ilde , betegnes ved Ordet : Velbehagelighed . H > Naar nu Christus saaled es er blev en din , og du , formedelst Troen , er ved Ham bleven reen ; naar du har faaet dm Arv , dit hoieste Gode , uden mindste din Fortjeneste , af Guds blotte <b>Kjarlighed</b> , som giver dig sin Son med al Hans Fortjeneste til Eiendom : da folge de gode Gjerninger , idet du gjor mod din Nceste , som du seer , at Christus har gjort mod dig . Her lares gode Gjerninger af sig <b>selv</b> . Thi siig : hvad er det vel for gode Gjerninger , Christus har ? ikke <b>sandt</b> , derfor ere de gode , fordi de ere strete til dit Bedste , for Guds Skyld , som bcfalede Ham at fuldkomme saadanne Gjerninger til din Frelse ? Deri har altsaa Christus varet Faderen lydig , at Han har tjent og elsket dig . Saaledes , naar du <b>selv</b> er bleven vederqvaget og beriget i Christo , har du kun eet Bud , hvori du stal tjeue og lyde Ham , dette nemlig , at du lader alle dine Gjerninger sigte til din Nastes Gavn . Derfor sagde Han ved Nadveren : Jeg giver Eder en ny Befaling , at I skulle elske hverandre , som Jeg har elsket Eder " ) . Saa prove da Enhver sig <b>selv</b> og agte noie paa , hvor nar eller fjern han er fra Christo , hvorledes det staaer tilmed hans Tro og <b>Kjarlighed</b> . Der ere vel Mange , som optandcs af en drommende Andagt , naar de hore tale om Christi Armod ; de vredes paa Borgerne i Bethlehem , straffe deres Blindhrd og Utaknemmelighed , og mene , at , hvis de havde varet i hincs Sted , saa vilde de have viist Herren og Hans <b>Moder</b> al mulig Tjeneste og ikke ladet det gaae saa jammerligt til . Men de see

<h4>1277</h4>Dig <b>selv</b> for os og ikke for Verden ? saa svarer Christus : Om Nogen elsker Mig , han skal holde mit Ord , og min Fader skal elske ham , og vi skal komme til ham og faste Bolig hos ham . " Nei , kjcere Judas ! — vil Han sige — det skal ikke gaae , som du troer . Verden har sin 3 Ere og Herlighed , sin Magt og Myndighed . Den vedkommer hverken Mig eller Eder ; men derom er det Mig at gjore , at man elsker Mig og holder mit Ord . Hvor man gjor det , der vil Jeg regjere , der vil Jeg aabcnbare Mig , der alene . Thi mit Regimente bestaaer ikke i at herske med Tvang og Magt ; nei , Hjerterne vil Jeg beherske , Jeg vil have Folk , som gjerne og villig komme til Mig . Det gjor ikke den Verden , der ei troer paa mit Navn . " Holde Christi Ord eller Bud , det maa Den gjore , som elsker Ham , det er , som erkjender og veed , hvad han eier i Ham ; thi hvo , som ikke vced det , elsker Ham ikke . Dette er da aldeles ingen Lov-Pra ' diken , men en Prcrdiken om den <b>Kjarlighed</b> og Naade , Christus har viist os , der Han tog vore Synder paa sig , hengav Liv og Blod for os , at vi stulle kjende Hans <b>Kjcerlighed</b> og gjore en salig Erfaring deraf , og ikke fordrcde andet af os , end at vi stulle troe dette , takke og elske Ham derfor , og saaledes , drevne af <b>Kjcrrlighed</b> , med Ord og Gjerning hjalpe til Hans Riges Udbredelse . At holde Christi Ord er ikke saa let en Sag , som de uerfarne Aandcr indbilde sig . Thi ei at tale om , at vort eget Kjod og Blod , vort naturlige Menneske , staaer imod af al Magt , saa er det jo vist og <b>sandt</b> i alle Maader , at man ikke kan begynde at bekjende Evangeliet , for Djcevelen er der med alle sine Lemmer og angriber de Troende med allehaande Forfolgelse og Aufcrgtning, indvortcs med Hjcrte-Angst og Sjcle-Kamp , udvortes med Verdens Had og Spot , saa man idelig maa raabe og strige til Himmelen om Hjcrlp . Da larer Erfaringen , at Christi Bud : Holder mine Ord ! har noget stort at bctydr . Derfor maa Hjertet — siger Christus — hange ved Mig og elske Mig . Hvo Mig ikke elsker , holder ikke mit Ord . <b>Kjcerligheden</b> kjcrmper og overvinder ; ja det er den rette <b>Kjcerligheds</b> Art , at den gjor Alt for den Elskedes Skyld , at Intet er saa tungt at lide og berre , at den jo gladelig gjor det . Det viser os allerede den naturlige Kjccrlighcd , der , som et Billede paa den guddommelige <b>Kjcerlighed</b> til os , er iudplantet af Gud i Fader og <b>Moder</b> til deres Born ; thi den brcender klar og stcerk i deres <b>Hjerte</b> og driver dem til , for Intet og mod dem , der stet ikke kunne siges at have fortjent

<h4>1826</h4>hvad jeg skal bruge derfor . Loven aabenbarer Sttndens Svgdom, Evangeliet ' giver Lagedommen . Det sees ogsaa klart af Dagens Tert . Den Skriftkloge tråder frem og sporgcr , hvad han sial gjore , for at arve det evige Liv . Loven giver ham det Svar : Du skal elske Herren , din Gud , af dit ganske <b>Hjerte</b> , af din ganske Sjel , af din ganske Styrke , af dit ganske Sind , og din Nceste ligcsom dig <b>selv</b> . Hvo , som laser eller horer dette saa lost hen , han forstaacr ligcsaa meget deraf , som den Skriftkloge . Nei , man maa trange ind i Ordenes Forstand og speile sig i dem . Skal jeg elske Gud af ganske <b>Hjerte</b> og Ncesten som mig <b>selv</b> , saa faaer jeg snart at see , hvor langt jeg staaer tilbage . Thi der maa da ikke vare et umildt <b>Blik</b> i mit <b>Die</b> , ikke et bittert Ord paa min Tunge , ikke en vredladen Bevagelse i mine Hander og Fodder , men min hele Person fra Isse til Fod , indvortes og udvortcs , maa gaae frem i <b>Kjarlighed</b> , have sin eneste Lyst i Gud og Hans Villie , og til alle Tider holde Hans Navn i LEre . Viser mig nu et Menneske , som er saa from , som har saadan Lyst og <b>Kjarlighed</b> til Gud ! Det kunne I ikke . Vi fole os langt tilboieligcre til Vrede og Had og til timelig Vellyst , end til Sagtmodighed , Reenhed og andre Dyder , og , saalange jeg foler en Gnist af ond Lyst hos mig , saa er hele Resten besmittet , og Loven er ikke fyldestgjort . Men nu sinder jeg ikke blot en Gnist , men en heel Ovn fuld af Syndens Ild i mig , og i mit naturlige <b>Hjerte</b> boer ingen <b>sand</b> <b>Kjarlighed</b> til Gud og Nastcn . Derfor seer jeg her i Loven , ligesonl i et Speil , at Alt , hvad i mig er , er fordomt og forbandet * ) , thi ingen Toddel af Loven stal forgaae , men Alt opfyldes , som Christus siger : Indtil Himmel og Jord forgaae , skal end ikke det mindste Bogstav eller een Tedde ! forgaae af Loven , forend de Ting skee allesammen"").

<h4>2089</h4>og Steender , f . Er . mellem Herre og Tjener , ja ei engang mellem Forceldre og Born ; thi her er <b>Kjcerligheden</b> ikke saa lige , ikke saa stcerk og fuldkommen paa begge Sider , her er Pagtcn ikke saa stedsevarcnde . Manden skal sorlade sin Fader og sin <b>Moder</b> og blive fast hos sin Hustrue , siger Herren ( 1 Moseb . 2 , 24. ) . As denne <b>Kjcerlighed</b> , denne hjertelige Tillid , folger nu ogsaa , at de dele Alt med hinanden , Alt , hvad de eie , Alt , hvad dem vederfares , vcere sig Godt eller Ondt den Ene tåger sig af den Anden , som af sit Eget ; den Ene staaer den Anden bi af yderste Formue ; gaacr det den Ene vel , saa er ogsaa den Anden glad ; lider den Ene , da grceder den Anden med. Dette stal vcere en Lignelse eller et Billede paa den store , hemmelige og vidunderlige Forening , som sinder Sted mellem Christus og Hans Menighed . Dens Lemmer ere alle de , som i <b>Sandhed</b> troe paa Ham : de ere af HansKjod ogßeen , ligesom <b>Qvinden</b> i Skabelsen er tågen af Manden . Det maa jo vcere en stor , ufattelig og uudsigelig <b>Kjcerlighed</b> , Gud bcerer til os , al den Stund dm guddommelige Natur forbinder sig saa usie med os , nedscenker sig saa dybt i vort Kjod og Blod , at Guds Son i <b>Sandhed</b> vorder eet Legeme med os , og tåger sig vor Jammer saa ncer , at Han ei alene vil vcere vor Broder , men ogsaa vor Brudgom , og giver os til evig Eie al sin guddommelige Rigdom , Viisdom , Retfcerdighed , Salighed , Styrke , at vi i Ham skulle vcere deelagtige i den guddommelige Natur . Og det er Hans Villie , at vi stulle tilegne os alt Dette i Troen , og med frimodig Tillid troste os ved denne Herre , som en Brud ved sin Brudgom . Saaledes er Hans Christenhed Himmelens og Jordens Dronning , thi den kaldes Hans Brud , som er en Herre over alle Skabninger , Hans , som paa det hoitideligste har forlenet sin Menighed Magt over Synd og Dod , Djcevel og Helvcde . See , dette viser Han os i hiint Billede , som vi dagligen have for <b>Die</b> , i Wgtestanden . Naar vi ere Vidner til fromme Mgtefolks <b>Kjcerlighed</b> og Trostab , eller naar vi see den Fryd og Gammen , som hersker ved et Bryllup , da stulle vi tcenke og troe : Christus har visseligen samme <b>Hjerte</b> og Sind mod Menigheden , som er Hans Brud , kun med den Forstjcl , at Hans <b>Kjcerlighcd</b> , Trofasthed og Naade er langt ftorre . Det viser Han os aabenbart ved sit Evangeliums Ord og sin Hellig-Aands Gaver , som Han fortener sin Kirke . Saaledes feirer Han den gladeste og herligste Fest , idet Han trolover sig med sin Menighed , slutter den i sine Arme , cercr og smykker den med alle sine Klenodier , med Syndens

<h4>2092</h4>Udslettelse , med sin Retferdighed , med sit Lys , sin Forstand , sin Styrke , kort , med Alt , hvad der horer til det evige Liv . Det er andre Kjeder og Ninge , andre Perler og Juveler , end disse jordiske , som kun ere et dodt Maleri af hine himmelske Klenodier. O , kjere Sjel ! oftlad da <b>Die</b> og <b>Hjerte</b> , og see , at du est Jesu kjere Brud , saasandt du af ganske <b>Hjerte</b> troer paa Ham ; Han indbyder dig til en herlig , en kongelig Bryllups- Nadvere : her er evig Fryd , Sang og Jubel , evigt Smykke og Herlighed , en Rigdoms Overstodighed af alle gode Ting ! Derfor stal ogsaa en hjertelig Tillid til Ham vore og tiltage i dit Inderste , al den Stund Han af usigelig <b>Kjerlighed</b> ved Daaben har kaldet og udvalgt dig til dette Samfund , antaget sig din Sag , forlest dig fra Syndens , Dodens og Satans Magt , vovet Liv og Blod for dig , ja hengivet sig for dig saa aldeles , at du ikke alene glad og frimodig kan rose dig af det , Han har fuldbragt for dig og skjenket dig , men ogsaa af Ham <b>selv</b> , som din egen . Og , ligesom en Brud med hjertelig Fortrostning forlader sig paa sin Brudgom , og anseer hans <b>Hjerte</b> for sit <b>Hjerte</b> , saa skal du af Hjertensgrund forlade dig paa Christi <b>Kjerlighed</b> , og ikke tvivle paa , at Han jo er sindet mod dig , som dit eget <b>Hjerte</b> . Men vceldigen strider herimod den gamle Adams , det er vort Kjods og Blods Blindhed og Forherdelse , som hverken lader os see eller troe dette , allermindst da , naar vi i os <b>selv</b> og i dette elendige Liv see og fole det Modsatte deraf . Vel seer og begriber min Fornuft , hvilket deiligt og frydefuldt Billed en Bryllufts-Hoitid og en Brude-Kjcerlighed i og for sig <b>selv</b> monne vcere ; den indrommer vel ogsaa til Nod , at Christus er en stjon , cedel , from og trofast Brudgom , og Hans Kirke en herlig , salig Brud . Men , naar jeg for min Person skal troe , at ogsaa jeg horer Christus til , at ogsaa jeg er et Lem paa Herrens Legeme , og at Christus bcerer saadan hjertelig <b>Kjcerlighed</b> til mig — da gaaer det ikke saa let . Det kommer deraf , at jeg ingen Herlighed seer hos mig <b>selv</b> , men derimod seer og foler megen Skrobelighed og Uverdighed , megen Trengsel , Moie og Sorg , ja Dod , Grav og Orme , som stulle fortere mig . Vil du nu seire i denne Kamp , da ler at troe det Ord , som Christus <b>selv</b> siger dig , og som Gud byder dig at troe . Vil du ikke gjsre Gud til en Logner , da maa du erkjende det for <b>Sandhed</b> , trods hvad du foler i dig <b>selv</b> . Thi stal du troe , da maa du ikke henge ved det , dine Tanker eller Folelser indstydc dig , men ved det , Guds Ord siger dig , om du end

<h4>2456</h4>sig med Haabet om Guds Forbarmelse . At det saaledes gik med Peder , og at han har holdt sig til Guds Ord og Naade , det vidne ogsaa Herrens Ord til ham hos Lue . 22 , 32. : Jeg bad for dig , at din Tro skal ikke afladc . Da han fornegtede Christus , da saae man ikke en Gnist af Tro i hans <b>Hjerte</b> ; men , da Samvittigheden vaagncde og Angst betog hans Sjel , da indfandt Troen sig atter og opholdt ham i Christi Ord , saa han ikke faldt i Fortvivlclse . Saalcdcs lcrrc vi da ogsaa her , hvori den rette Omvendelse bestaaer . Peder grced bitterlig . Det er Begyndelsen til Omvendelse , at Hjertet ret erkjender sin Synd og oprigtigen hader den , saa man ei har Lyst eller <b>Kjerlighed</b> dertil , ei heller fremturer derudi , men inderlig bedre v es , fordi man har krcrnket Guds Villie . Denne Erkjendelsc og Anger kunne vi ikke <b>selv</b> frembringe , men den er en Frngt af , at Herren kalder os og seer til os igjen . Saaledes kaldte han Peder tilbage ved Hanens Galen og ved sit eget <b>Blik</b> , som Han kastede paa ham . Men , naar vi nu erkjende Synden , da kan den ikke andet , end gjere Hjertet bange og bekymret ved Forestillingen om Guds Vrede . I saadan Angst og Hjerte- Sorg skal du da allerferst ydmyge dig for Gud , erligen bekjende dine Synder og bede om Naade for Jesu Christi Skyld . Derncest stal du holde dig fast ved Guds Ord og Forjcrttclse , og visselig troe , at Gud for sin Sens Skyld vil vcere dig naadig . Da viger Angsten , en salig Trest oprinder i Hjertet , og den sande Omvendelse er steet . Denne Trest var hos Peder i Begyndelsen meget liden , som et Seneps- Korn ; men saasom Guds Ord og Forjettelse var den Grund , hvorpaa denne ringe Trest var bvgget , saa vorede den , og paa Paaske-Dag , da han saae Christum igjen , var al Angst og Forferdelse borte , og kun en hjertelig Idmyghed var bleven tilbage . Trosten havde da , ligesom en sterk Negn fra Himmelen, dempet og udslukket den Smertens Ild , som forhen truede med at fortere hans <b>Hjerte</b> . Saa staaer det da fast : vil du vere en <b>sand</b> Christen , saa frygt Gud , og fortro dig til Hans Naade og Ord , saa sinder du Trost , Frelse og H ^ elp her og hisset . Det forlene os Alle vor kjere Fader i Himmelen ved sin Hellig-Aand for vor kjere Frelsers Skyld ! Amen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016042848154'target=_>Ahlfeld, Friedrich, 1866, Prædikener over Kirkeaarets Evangelier</a><h4>540</h4>for ung . I Nazareth var det bedre . Gid jeg dog var bleven hjemme ! " Han drager med , han drager med ligetil den hellige Gudsstad . — Men hvor Mange af os ere ikke blevne liggende paa Halvveien eller paa Fjerdedelsveien og ere saa atter uformcerket vendte tilbage til den gamle døde Ros gamle Hus ! — Tung , mine Elskede , er vistnok ogsaa vor Vei . Men du har de samme Midler til at lette dig den , som hine havde . Tcenk dog flittig paa de store Gjerninger , som Gud har gjort mod dig , mod dine Fcedre , mod hele Kirken . Er da ikke hele Kirkens Historie et Canacm , beskrevet og betegnet med tusind Seire , som Herren har vundet over sine Fiender ? Er den ikke en Bog , af hvilken enhver Side boerer Mærke af hans Trofasthed ? Er ikke selve dit Liv et Blad i denne Bog ? Kommer du da under dine Tanker paa dig <b>selv</b> og paa Kirken til et redt Hav , gjennem hvilket Gud slog Vei , til en Klippe , som maatte give Vand , til en Gideonskamp med trehundrede Mand mod utalte Fiender , da syng en af vor Kirkes gamle Seierssange . Vi have Hsisange nok . O det hjælper godt under Vandringen. Da kommer man over de tunge Steder , man ved ikke <b>selv</b> hvordan . — Du har endogsaa en Ledsager , en Hjælper paa Vandringen , som hine vistnok ogsaa havde men ikte saa . Og du ser ham ; du maatte være blind med sunde Vine , hvis dn ikke vilde erkjende , at Christus gaaer med dig . Midt i Syrerhceren aabnede Herren Elisas bange Tjeners <b>Oine</b> , saaat han saa Herrens Hoer , der er mægtigere end nogen Fiendehcer . Maatte han da ogsaa aabenbare sig for dig paa din Pilgrimsvandring , saaat du overalt ser ham , der med Rette hedder „ Israels Vogne og Nyttere " . — Naar du drager op til Zion , saa drag ikke alene . Joseph tog Maria med. Mand og <b>Hustru</b> skulle tilsammen valfarte til den hellige Stad . Du Mand , hvis Vine Herren har aabnet , for hvem han har borttaget det gamle Forhceng , du som har det rette Maal foran dig , sig til din <b>Hustru</b> : „ Kom , vi ville gaa sammen ! " Du , <b>Hustru</b> , hvem Herren har givet et nyt <b>Hjerte</b> , bed din Mand : „ Kom , lad os gaa sammen ! " Hvad er da den hellige Wgtestand ? Er den blot en Pagt om tyve eller tredive Aars fcelles Arbeide , om tyve eller tredive Aars fcelles Glæde og fcelles Bekymring for Brod , Helbred og Bsrn ? Da saa Ingen , at denne Stand stammer fra Paradiset . Dens Hovedoiemed er det , at den Ene skal tage den Anden med og forfremme ham paa Frelsesveiens , paa Veien til det Jerusalem , som er heroventil . Mange Mænd have ogsaa opvakt fine Hustruer til den hellige Tro . Mange Hustruer have med Bon , <b>Kjærlighed</b> og Taalmodighed bragt sine Mænd fra det forfcengelige Væsens Tjeneste til lesn Christi levende Efterfolgelse . Det kan ogsaa lykkes Eder . I Ynglinge , I Jomfruer , for hvem Herren endnu har ladet

<h4>1092</h4>hos Gud . At de ere dobte , det kunne de ikke for . De ere ikke netop bedrovede over , at det er steet ; men de have heller ikke en Gnist af Gloede i Hjertet derover . Det er nu engang gjort . De ere ikke Christi Fiender i den Forstand , at de skulde ville odelcegge hans Kirke . Men de ere heller ikke hans Venner . Ikke et Gran varmere eller gladere slaaer deres <b>Hjerte</b> , naar det forestiller sig det store Offer , som han har bragt for os , den store <b>Kjærlighed</b> , hvormed Guds enbaarne Son har tjent os . — Naar man taler med dem om de Gjerninger, som Christus har gjort , saa finde de vistnok hans Undere store nok . De ville ikke have nogen storre . Men en anden Betænkelighed opstiger hos dem : „ Det er saa lcrnge siden han gjorde dem . De siede i et fremmed , , fjernt Land . Jeg var ikke tilstede og saa dem ikke med egne Vine . Der blev ' heller ikke optaget nogen Protokol derover af Notar og Vidner . Ja , » havde jeg blot kunnet være Vidne til et eneste af dem ! Havde jeg blot staaet derhos een Gang naar han opvakte en Dod , aabnede enßlindsDine og bespiste hungrende Skarer ! " Vid : havde du endog staaet derhos , du troede dog ikke . Troer du ikke fire Evangelister , saa vil du heller ikke tro dine to Dine . Naar Tvivlen afkrcefter fire guddommelige Hjemmelsmand , saa finder den ogsaa Indvendinger mod to Vine . Troer du ikke den hele Kirke , som er bygget paa Christi Liv og Dod , paa hans Ord og hans Undere , saa troer du heller ikke dine egne Dine . Dit indre <b>Die</b> er jo et ondt <b>Die</b> , derfor kan heller ikke det ydre se ret . — Da den rige Mand i Helvede beder for sine Brsore , siger Abraham til ham : „ De have Moses og Propheterne ; lad dem hore dem . Hore de ikke Moses og Propheterne , stulle de heller ikke tro , om Nogen opstod fra de Dode " , Hine , som forlangte et Under fra Himlen , kunde man svare : „ Det er jo netop Undere paa den rette Himmel . Herren gjor sine Gjerninger ikke alene paa <b>Die</b> og Dre , Fod og Haand , han gjor dem ogsaa paa Hjertet . Men ethvert Under paa Synderens <b>Hjerte</b> er et Under paa Himlen . I enhver fra denne Verdens Fyrste vunden Sjæl scetter Jesus sig en ny Stjerne paa sin Himmel . Det er de første Undere " . — Men du , som savner Undere i Kirkens nærværende Tid , du savner dem kun , fordi du er bliud . Vel er det <b>sandt</b> , der har været fattige , torre Tider i den evangelists Kirke . Som stakkels Eneboere have de levende Christne boet i Verdens vilde Skove . Men <b>selv</b> i disse sorgeligste Tider har det ikke manglet paa store Naadeundere af Gud . Ogsaa da ere dove Dren blevne aabnede , ogsaa da ere blinde Dine blevne gjorte seende . Men fordi du <b>selv</b> er forkommen i Kjedets Tjeneste , gjælde ogsaa kun Gjerninger i Kjodet dig for Undere . Naar Herrens Aano har trukket og strcekket ret en i Morke og Synd forkommen og

<h4>1361</h4>Den anden er <b>Kjærligheden</b> som doer for os . Det er din Herres og Frelfers <b>Kjærlighed</b> . Lad os vende <b>Blikket</b> fm Himmelshsiderne ned til Korset paa Golgatha . Lad os vende <b>Blikket</b> op fra det gnddommelige Naads Dybder til den Kjærlighedsdaad , som Ssnnen fuldbringer for os . Lad os betragte denne <b>Kjærlighed</b> i dens hele Omfang . Ligesom Herren udbreder sine Arme til begge Sider , faaledes udbreder han sine Kjærligheds Arme mod den hels Verden . Og udstrcekker han end her sine Arme tvungen af raa Vold , faa udbreder han dog sine Kjærligheds Arme af sin egen Villie og i hellig Frihed . Han udbreder dem over dem , fom græde omkring ham , som endnn have bragt en Levning af sin gamle <b>Kjærlighed</b> med til Korset , hvem <b>Kjærligheden</b> har fort til Hovedpandesteoet . Han udbreder dem efter dem , som staa der slove og ligegyldige , hvem Nysgjerrigheden har fort til Hovedpandesteoet . Han udbreder dem efter dem , fom bragte ham i Doden , hvem Hadet har fort til Hovedpandestedet . Ja , var Judas , som forraadte ham for 30 Solvpenge , istedetfor til Templet kommen for denne Naadestol , og havde kastet ikke Solvpeip gene men sig <b>selv</b> med Sjæl og Legeme for hans Fodder , — han havde udbredt sin forbarmende <b>Kjærlighed</b> ogscia over ham . Han beder for sine Fiender : „ Fader , forlad dem , thi de vide ikke hvad de gjore ! " Han udbreder dem efter sin <b>Moder</b> og efter den Discipel , som han elskede ; han udbreder dem ogsaa efter de Misdcedere , som hang ved hans Side ; men kun een griber dem . Han udbreder dem over Israel og Hedningeverdenen . Vi fole det : paa hint Sted faldt der en Velsignelse paa mange Abrahams Sonner og Detre ; der faldt ogfaa en Velsignelse paa den romerske Hovedsmand , som holdt Vagt med sin Skare . Han udbreder dem efter dem , som have levet for ham , og efter dem , fom stulle leve i Fremtiden . De , som i Dodens stille Nige ventede paa , at deres Forlosning skulde nærme sig , dem indeslnttede han i sin <b>Kjærlighed</b> . De , som endnu ikke vare fodte , dem bar han paa sit <b>Hjerte</b> . Israels Ippersteprcest bar Israels tolv Stammer indskrevne paa sit Vryststjold . Christus , denne sande Dppersteprcest, bar alle Jordens Folk og alle enkelte Sjæle indskrevne dybt i sit <b>Hjerte</b> , ikke i Sten , men i den levende <b>Kjærlighed</b> . — O kjære Vrsdre , naar det er koldt , trcekker Kulden Alt sammen , f . Ex . dine Fingre , saaat din Trolovelsesring kunde glide af dig . Naar Nodens

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009082403016'target=_>Linderot, Lars, 1864, Lars Linderots fuldstændige Huus-Postille</a><h4>3046</h4>sine Borns Behov .  Ak see " , raaber Frelseren,  o see , du Lidettroende , som sorger saa overdrevent for den fornodne Fode , see paa Fuglene under Himmelen , som hverken saae eller hoste , eller samle i Lader , og de forsorges dog af den barmhjertige himmelske Fader ; men du , som er skadt i et hsiere Viemed og stal gjenlsses med Sonnens Blod , du gruer uvcerdigen for det jordiske Ophold ! O , er du ikke mere end alle Fugle ? Ligesaa du , siger den Herre Jesus , som gaaer i kvcelende Bekymringer for de nsdvcndige Klceder ; kast et eneste opmcrrksomt <b>Blik</b> paa de stumme Blomster paa Marken , og de skulle bringe dig til at blive stamfuld . Hvem kloeder dem , da de hverken , som Mandfolkene idlandt os , arbeide , eller spinde , hvilket for det meste er blevet <b>Kvindernes</b> Lod , og dog forsikkrer jeg , at ikke <b>selv</b> Salomo i al sin kongelige Pragt , som udmcerkede ham paa hans Throne , var saa majestattist kloedt som en eneste Lillie paa Marken ; iscer da han prunkede med en laant Dragt , men Blomsternes Prydelse er deres egen . Har nu Gud formaaet . at scette saadanne Klceder paa Grasset , som dog ikke benyttes til Andet , end til at kastes som Brcendsel i Ovnen , hvor skulde han da kunne negte Eder , som ere bestemte til Salighed , de Klaeder , som udfordres til det arme Legeme ? " Saadan var Frelserens skjsnne Slutning i denne vigtige Deel af Helliggjørelsen ; men nu til noget lllmindeligere Bemerkninger om Helliggjsrelsen. Vi llrre da , at Helliggjorelse nodvendig stal finde Sted , at der til en <b>sand</b> Helliggjerclse stal hentes Kraft Na Christi Forsoning , og at den bedste Helliggjorelse hos de redeligste Guds fenner dog er ufuldkommen paa Jorden. Forst paastaae vi for den levende Guds Myn , at Helliggjorelse nodvendig maa stnde Sted . Sagens Natur taler for denne Paastand . vanhelligt <b>Hjerte</b> og en vanhellig " andel passe unegtelig ikke for noget andet

<h4>3261</h4>Tale for dig , som du ikke gjerne vil hsre . Jeg veed , at du troer ikke dette Herrens Ord ; nei , du mener , at det er bedre med dig , end det virkelig er , du mener , at det er vel , og det er ilde . Men tcenk , om der sad et Vindue paa dit <b>Hjerte</b> , saa at du kunde ste ind i dit <b>Hjertes</b> Dyb , ak , du skulde faae see saadanne Ting , at du skulde falde i den yderste Forskrekkelse og Bestyrtelse ! O , om dog den Helligaand kunde faae aabne Vinene paa dig ! O , om han fik aabne dit <b>Hjerte</b> i denne Stund , saa at du fik kaste et <b>Blik</b> ind i dette skrekkelige Dyb , du skulde faae see , at det er et forfcerdeligt Gods , som ligger i dette Skatkammer. Da skulde du give Gud Ret , at han har talt <b>sandt</b> gjennem Kong Davids Mund , naar han kalder dig en Vederstyggelighet Men David gaaer videre , naar han stal beskrive Synden . Han siger ogsaa , hvordan de uomvendte Syndere ere imod deres Nceste . Intet due de med Hensyn til sig <b>selv</b> . En Vederstyggelighed ere de for Gud ; og der er Ingen , som gjor Godt imod sin Nceste . Jo , jo , du beklagelsesvcerdige Sjcel , som blot staaer i din Natur , i dit ikke dsdede Kjsd , i den gamle Adams Hud ; sandelig for den levende Gud , du er ikke bedre imod din Nceste , saa lcenge der ikke bliver nogen Forandring i denne Tilstand . Kom og betragt dig nsie her , saa stal det blive aabenbart for dig , hvor gruelig du har syndet imod Ncesten . Eller er ikke det en stor forbandet Synd , at du , arme ulydige Barn , kan vcere saa gjenstridig imod alle gode og christclige Formaninger ? Er ikke det en stor forbandet Synd , at du syndige Fader og <b>Moder</b> skal vcere saa ligegyldig for dine arme Berns dyrekjsbte Sjcele , at du just med al Flid opammer dem i Djevelens Tjeneste ? De lcere sig til at svcerge , naar de hsre dig raabe paa Djevelen ; de lere sig til at lyve , naar de hore dig fare med Lsgn ; de lere sig til Bitterhed , naar de see dig at vere vred og opirret ; de lere sig til Utugt , naar de hsre dig drive utugtigt Skjemt ; de lere sig til at sette Priis paa jordisk Gods , naar de hsre

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016063008096'target=_>Luther, Martin, 1831, christelig Postille</a><h4>85</h4>flere Tjenere , som saa at sige kunne være Haardt mod Haardt og Alvor mod Alvor , ligesom der til en knudret Green horer en haard Icrnkile . Og naar Gud lader et Uvcir rase , saa man ikke alene dermed folge Regn for at bcfngte Alt , men der maa og folge Torden , som kan stette Alc i Bevægelse , og et Lynstag , som kan rense Luften , paa vet at Jorden kan beere saameget des flere og bedre Frugter . " Af Luthers private Liv i Aarene 4625 — 29 ville vi ister lcegge Mærke til tvende Begivenheder . Den første er hans Gift termaal med en forhenværende Nonne , Katharina von Bora , til hvem han den 13 de Juni lod sig vie af Dr. Pomme ? ranns : og blev han i dette 2 Egtestab en saare om og trofast g tem ag c , og en <b>sand</b> Fader for sine Born , hvilke han elskede med den inderligste <b>Kjærlighed</b> og , efter Guds Ord , opdrog i Tugt og Herreus Formaning . Overhovedet ere der vel Faa , der med et saa christcllgt <b>Blik</b> have betragtet saavel det hu uslige, som borgerlige Livs Forhold , i hvilke han stedse saae og ccrede Guds lud stiftelse , hvori altsaa Ingeu maatte om ? gaaes efter sin , men efter Herrens Vittie . Den anden Begivenhed, vi ville betragte , er den haarde Sjels og Legems Sygdom, udi hvilken han nogle Aar efter faldt . Under denne led han decls af legemlige Smerter , ister af en med strcekkclig Pine forbunden Susen og Brusen i Hovedet , dcels af aandelige Anfægtelser, Tvivl , Frygt for Fordommelse , maaffee ogsaa « andelig Hovmod o . s . v. , i Anledning af hvilke sidste han siger iet af sine Breve : mener mig det med Alvor og rafcr med utrolig Vrede og Forbittrelse . Det er underligt , hvorledes han førstiller sig , som ChristuS <b>selv</b> , ikke at omtale som en Lysets Engel ; han anvender al sin Kunst paa mig for at styrte mig i Fordceevelse . " len sterdelcs pinefuld Time talte han saaledes til Herren , efter af inderste <b>Hjerte</b> at have bedet Fader-Vor og den tite Davids Pjalme : " Herre , allerkæreste Gud ! Ak , hvor gjerne havde jeg udgydet mit Blod for dit Ords Skyld , det vced Dn ! Men jeg er maastee ikke dertil vcerdig : din Villie ffee ! Vil dn det saa have , da vil jeg gjerne doe , kun at dit hellige Navn bliver lovet og vriset , det være cuteu ved mit Liv eller min Død . Men , var det mnligt , da vilde jeg gjerne endnu lccngere leve for dine Gudfrygtiges og Udvalgtes Skyld : er derimod Stnnden nu kommen , da g ) or , som det Dig behager ! Dn er en Herre over Liv og Død . Min allerkæreste Gud ! Du har jo fort mig ind i Sagen ; Du vecd det , at der er dit Ord og diu <b>Sandhed</b> . Ophoi ikke , glæd ci heller dille Fiender , paa det de ikke stulle rose sig , sigcude : hvor cr uu dcrcS Gud ? men forklar dit hellige

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011072120001'target=_>Walnum, S.B. Hersleb, 1880, Prædikener over Kirkeaarets Evangelier</a><h4>448</h4>dine Dine , og see ! de samle sig allesammen ; de komme til dig ; dine SMncr , de skulle komme fra det Lierne , o ' g dine DjNtre , de sknllc bceres paa Armen . " Idet Gnds Menighed skner rundt omkring i den vide Verdcn , skal den see , hvorledes Hedningerne forsamle stg , for at komme til den aandelige Fredsstad og at nyde dens bimmclste Goder ; og DMtre as hedenske skulle indkomme i Guds Menighed og opfostres til Guds Wrn baarne , pleiede og ledede af den aandelige <b>Moder</b> Guds Kirke . ~ Da skal du see det og straale af Glcede , og dit <b>Hjerte</b> skal banke og udvide sig ; thi Havets dom skal vende sig til dig , Hedningernes Gods ska komme til dig . En Mangfoldighed as Kameler stal bedcekke dig ; Midians og Ephas nng e Kameler , allesammen skulle komme fra Scheba ; Guld og Virat kulle de bcere , og HerrensPriis skulle de forkynde <b>Hjerte</b> skal banke af Fryd og udvide stg as naar man seer , hvorlcdcs den store Skare , som bor ved Havet vender om fra deres Veis Vildfarclse og Herren , hvorlcdcs den Mangfoldighed , som drager frem paa Kamcler , kommer til Frcdsfyrsten , hvorledcs de bringe ham deres Gåver og Hjerters glade Lovsange , ogsaa de , som fM havde vceret Israels og Herrens Fiender . < , « See , hvad Herren i disse Ord har lovet , det har han iaiennem Tidernes Lob eftcrhaanden fuldbyrdct Han har fr mkaldt , dyqtiggjort og udscndt de <b>Sandhedens</b> Vidncr , som ans Navu gik ud til Hcdningefolkene for at e dem Korscts Ord , og at indbyde Sjcelcne tll BryllupshMdcn i Guds Rige . Han har <b>selv</b> beredct Hjerterne dcrudc saa at de blcve en lordbuud , hvori Ordets gode Sced kunde Pire og bcere Frugt . Han har i den bcleilige Tune og aa vde formorkedc Hednmger udtalt sit almcegttge Ord : ~ VUv Lys ! oa der blev Lys " ; thi Herrens Naadelys skmncde md rde som ikke valgte at vandre i Merket ; <b>Sandheden</b> r Christo frigjorde de Bundne ; Herrens og Fred scenkede sia ned i de bange Hjerter ; Herrens Opstandelscs Kraft mcegtigen i de Skrßelige ; Guds <b>Kjcerligheds</b> Sol fremkaldte en deilig Sommer med Varme og Lw i Hlerterne bvor der fsr var aandeliq Vinter med isncnde Kulde og Dpd og alle de Sjcele , som fandt Lyset og Livet i Christo , d ? have med jublende forkyndt Herrens Pr.is . Men liaesom Herren hidindtil har udseudt stue Frcdsbud til de arme Hcdningefolk , saaledes vil han og vedbkve dermed

<h4>483</h4>LMe , at Ingen stal rive diss ud af hans Haand ( Joh . 10 , 28 ) ; du har i Fristelsens Stund erfaret , at han aabenbarer sin Kraft mcegtig i din SkrFbelighed ; du ~ er forsikkrct om dette , at han , som har begyndt en god Gjerning i os , vil fuldfM den indtil Jesu Christi Dag " ( Philp . 1 , 6 ) . Naar du saa under Aarvaagenhed og BM holder dig ncer til din Frelser og styrker dit Troesliv " ved Guds Ordbog Nadveren , saa vil det nye Menneske i dig ~ forfremmes i Viisdom og Alder og Naade hos Gud og Mennesker ; " thi Guds Aand vil stadigen forklare Guds Ord for dit <b>Hjerte</b> , saa at du gaar stem fra Klarhed til Klarhed og bcdre og bedrc forstaar , ~ hvad der er Guds gode og velbehagelige og stildkomne Villie " ( Norn . 12 , 2 ) ; Guds Aand vil hjcelpe dig til inderligere og inderligere ved Trocn at tilegne dig Naaden i Christo , og at styrkcs i Hjertet ved den , saa at Intet kan stille dig fra Guds <b>Kjcrrlighed</b> i Christo Jesu . Da ~ opvoxer du i alle Maader til ham , som er Hovedet , til Christus " ( Eph . 4 , 15 ) , og du vil vinde Indest hos allc.dem , som ere af <b>Sandheden</b> ; da siger du freidigen : ~ leg slipper lesum ei ; med Jesu vil jeg kjcrmpe . " Men idet du fMr denne Kamp , kan du ofte blive dMtrcet, og for det gamle Menneske hos dig vil din Vci synes saare tung og tornestrM ; du vil erfare <b>Sandheden</b> af det Herrens Ord , at den , som vil Me ham , faar sit Kors at bcere ; men saa erfarer du ogsaa , at ~ han lindrer dig dit Kors med TrD i rette Tide , " og derfor synger du af Hjertet <b>selv</b> i Smertens bittreste Time : ~ leg slipper Icsum ei ; med Jesu vil jeg lide . " Naar saa endelig den sidste Kampens og Lidelsens tunge Time oprinder , naar Fyrste for sidste Gang udfiyder sine gloende Pile imod dig , da stirrer du med Troens <b>Die</b> paa ham , den Frelser , som smagte Dvdens Bitterhed for dig og seircdc over og Grav , og glad og frimodig i ham siger du da :

<h4>498</h4>denne hans kjcerlige Hensigt opnaaes ikke af alle Mgtefolk , dcels fordi Mand og <b>Hustru</b> leve i Syndens Tjeneste , deels fordi den Enc lider for den Andens Synd ; thi ligesaavist som en from og dygtig Mgtefcelle vil bcrede dig stor Glcede og kan bidrage saare megct til at fremme baade dit timelige og dit aandeligc Vel , ligesaavist kan en ugudelig og uduelig Mgtefcelle bringe dig stor Sorg , kan Fdclcegge din timclige Velfcerd og maaskce drage dig langt bort fra Herren . Derfor er det uendclig vigtigt for dig , som vil indtrcedc i Mgtestanden, at faae en Mgtcfcelle , som baadc kan og vil fremme dit timelige Vel , men som frcmfor Alt baade kan og vil leve sammen med dig i Herrens Samfund , baade kan og vil vcerc til Hjcelp og StMc for din Christentro og Gudsfrygt . Vcer derfor yderlig forsigtig i Valget af dit Livs Ledsager , og fKg ikke det Exempel , som Mange iblandt os give ! De sFge ikke Veiledning hos Herren , ak ! ham have de forladt , og om hans Villie bekymrc de sig ikke ; de see ved Valget af deres Mgtefcelle kun efter ndvortes SkjMhed eller efter jordiske Skatte eller efter Stand ; de staac med en paa deres Lceber , naar de for Herrens Alter svare Ia paa det om ~ de have beraadt sig med Gud i Himmelen , med deres egct Hjertelag og med deres Slcegt og Venner . " Og som de have gjort deres Valg uden Gud , saaledes stutte de ogsaa den hellige Pagt uden Gud ; men saa faae de og siden ofte den bittre Frugt af deres Letsindighed , idet deres Mgtestand bliver dem en Sorgens Stand , og Forbandelsen liggcr tungt paa dem . Derfor — vil du ikke udscette dig for at gjprc den samme ftrgclige Erfaring , o saa dig ~ mcd Gud i Himmelen " , idet du stal vcelgc dig en Mgtefcelle ; bced ham alvorligcn om Naade til , i dette dit Livs vigtige Anliggendc at forstaae hans Villie og at boie dig ydmygeligcn under den ; beed ham indcrligcn om , at han vil lede dit Valg , saa at du maa erholde en god og trofast Mgtefcelle ! RaadfFr dig saa ~ mcd dit cgct <b>Hjerte</b> " ; uudcrssig , om du derinde ncerer <b>sand</b> Agtelse for og <b>Kjcerlighcd</b> til den , hvem du tcenker at vcelge til Mgtefcelle , saa at du haabc , at I trofast ville ~ dele Vee og Vel med hinandcn , i hvad Lykke Gud almcegtigstc vil Eder tils > ic . " Raadfpr dig saa , hvis du dertil har Anlcdning , ~ med din Slcrgt og dine Venner " , og da frcmfor Alt med dine Forceldrc ; thi de have mere Livserfaring end du <b>selv</b> , og de ville saa gjerne fremme dit Vel ; deres Velsignelse over dig og over din Mgtcpagt er i <b>Sandhed</b> en god Gave at tåge med i Mgtestandcn ; ~ Faderens Vel-

<h4>1498</h4>~ Gud har elsket Verden " ; ligesom Faderen og SMnen har elstet Vcrden med en ncndelig stor <b>Kjcerlighcd</b> , saaledes ogsaa den Hellig Aand , som er <b>sand</b> Gnd med Faderen og SMnen . Hvor klart aabenbarer han os ikke sin <b>Kjcerlighed</b> i store Naadesgjerning ! See , vi formaac ikke <b>selv</b> at opfylde Betingelsen for , at vi knnnc blive deelagtige i den os forhvervede Frelse ; vi kunne ikke af vor egen Styrke eller Fornuft troe paa Christum eller komme til Christum, vor Herre , og vi formaae heller ikke ved vor egen Styrke at blive i Troen indtil Enden ; men det er den Hellig- Aands Gjerning . Ved Daaben begyudte han sin Kjcerlighedsgjerning i vort Barnehjerte , for at vi ikke i nogen Alder stulle vcere udelukkede fra Christi Samfund og Naade ; siden fortsatte han den igjennem hele vort Liv . Ved Loven han at oplyse os om vor Synd og Guds Vrede over Synden , for at vi stulle fple Hunger efter Naaden ; ved Evangelium han at oplysc os om Guds <b>Kjcerlighed</b> i Ehristo , og aabncde vi Hjertet for ham , da hjalp dan os til i bevidst Tro at tilcgne os Naaden i Christo . Siden vaagede han over os Mmere, end en <b>Moder</b> kan vaagc over sit Barn . Han styrkcde vor Tro , vort Haab og vor <b>Kjcerlighed</b> og bevarede det nye Liv i os , idet han ved Ordet stadigcn veiledede os til Sandhcdens Erkjcndclse , og alt klarere og klarere aabenbarede os Guds Nigcs Hemmeligheder , idet han bcrededc vort <b>Hjerte</b> , saa at vi ved Nadvcren kunde smage vor Frelsers <b>Kjcerlighed</b> og mcegtigen befcestes i Livssamfundct med ham . Han tugtede os igjennem Samvittigheden , naar vi bleve lunkne eller ulydige imod Guds Ord , og vakte Uro og Smerte derinde i Hjertet , for saa at triste os livsaligt ved Ordet om Guds inderlige Kjcerligbed . Han bestjermede os imod KjFdets og Verdens og Djcvelens Fristelscr , idet han lcdsagede os med sin hemmelige Paamindelse og Advarsel og bragtc os vor Herres Jesu Ehristi Seierskraft . O , seer dog , hvor stor en <b>Kjcerlighed</b> den trofastc Hellig Aand beviser os , idet han saaledes utrcettcligen arbeider paa at bringe Menncstene til den sande Tro , uagtet de ofte ere gjenstridige imod ham , saaledes utrcettcligen arbeider paa at lede de Troende frem paa SaliggjFrelsens Vei , uagtet de ofte ere dorske til at lade sig lede af ham !

<h4>1747</h4>agtc ikke paa , hvad Herren har sagt : ~ Hver den , som seer paa en <b>Qvinde</b> , for at begjcere hende , har allcredc bcdrevet Hor med hende i sit <b>Hjerte</b> " ( Math . 5 , 28 ) . De give maaskee mange Almisser til Fattige ; de tunne virke gavnligt i Borgersamfundet; men dette gjFre de ikke as <b>Kjcerlighcd</b> til Gud , ikke i villig Lydighed under hans Villie , men for at vinde Anseclsc hos Mennestene eller for at fortjcne sig det evige Liv hos Gud . De see ikke ocres Naturs dybe Fordcervclfe og ikke deres store Syndestyld ; derfor indbilde de sig , at de crc vcerdigc til at arve Guds Rige . Men , I forblindede Pharisceere ! I bedrage Edcr <b>selv</b> med et i Bund og Grund falskt Haab ; Eders Egenrctfcerdighcd er som et sMderrevet og besmittct Klocdebon ; den kan ikke skjule Edcrs Skam for den Hclliges st ) ine , og Herrerncs Herre har dMt , at med den ~ komme I ingcnlundc ind i Himmerigcs Rige . " Men der er Andre , hvis Hjerteforhold til Herren er ganske andcrledcs end de Skriftklogcs og Pharisceerncs ; det er de Vakte . De crc ved Herrens store Naade opvaagncdc af og erc , ligesom den forlorne SM , ~ komne til sig <b>sclv</b> " ; de deres Elcndighed og have faaet opladt for Syndcn som dens rette Aarsag ; de bceve for Guds Vrede og Trang til Retfcerdighed for ham ; under VM og flittig Brug af <b>Sandhcdcns</b> Ord stride de imod Synden og beflitte sig paa at adlyde den Hclligcs Villie . Have da de den Retfcerdighed , hvormcd de kunne bcstaae for Gud ? Nei ! de formaae ikke at udslctte hele deres forcgaacnde Livs store og forfcerdelige Syndeskyld ; de lcegge frcmdeles Skyld til Skyld ; thi de mangle den <b>Kjcerlighed</b> , som er Lovens Fylde , og derfor kunne de ikke vise den fuldkomne Lydighed , som Loven krcevcr ; den syndige Lyst i deres KjFd trceder dagligen frem i ondc Tanker , Ord eller Gjerning . De have vistnok gjort et overmaade vigtigt Skridt frem paa Vcien til Guds Rige ; men faalcengc de ikke ved en levende Tro paa den Korsfcestede have tilegnet sig Naaden og Frclsen i hans Navn , saa ere de ikke sande Lemmer i Naadcns Rige og crc ikke blevne retfcerdiggjorte af Gud . O , bctcenkcr dette vel , I vakte Sjcele ! og vcerer aarvaagnc over Edcr <b>sclv</b> , at I ikke stulle af Sjelefienden blive forførte til en fim Hovmod og atter drages ind i MMkct ! Ak ! der gives ocm , som blive staacnde i Opvcekkelsens Stand , mcdens denne stal vcerc en Gjcnnemgang til at blive Guds frimodige ved Trocn ; de ssige en Slags TrFst i Angerens Smerte , og i Hjertcdybet liggcr stjult den

<h4>1921</h4>skulde staae ved Siden af Toldere og arme Syndcre ; paa dem saae han ned med dyb Foragt ; derfor fremstillede han sig for Guds Ansigt meget tilfreds med sig seld , og han glemte , hvad Herren har sagt : ~ Den , som scetter Dinene HM og har et hoffcerdigt <b>Hjerte</b> , ham kan jeg ikke fordrage " ( Salme 101 , 5 ) . Han kunde derfor ikke med Toldercns ydmyge Sind bede : ~ Gud , vcer mig Synder naadig ! " Have vel vi allesammen lcert at bede Tolderens BM med ydmyge Hjerter ? Det kunde synes , som var det let for os at komme til en levende Syndserkjendelse ; thi fra vore Barndoms Dage kjende vi de 10 Bud , i hvilke Herren forkynder os sin hellige Villie , og vi have tillige vor Frelsers klare Ndvikling af Vudenes dybe , aandelige Betydning ; vi ere af Forceldre og Lcerere saa ofte blevne mindede om vor Naturs dybe Fordcervelse og om vor mcgen Nlydighcd mod Fader og <b>Moder</b> og mod vor Gud ; vi have tillige i Guds Huus HM saa mange straffcnde Ord om vore grovere eller finere Synder , og hvor ofte har ikke Herren bebreidende talt til os igjennem Samvittigheden ! Og dog er der Mange iblandt os , som i Sindelag ligne hin Pharisceer . De cre blinde for Guds Hellighed og for hans strenge Krav i Loven , blinde for deres Hjerters store Urenhed og Mangel paa den <b>Kjcerlighed</b> , hvorved Herrens Bud kan opfyldes , blinde for den store Synd , som de have bcgaaet ved deres ukjcerlige DMmen og deres bittre Tale mod Ncesten , ved deres mange usandfcerdigc , letsindige og bespottelige Ord , blinde for de grovere eller finere Synder , hvori de leve , blinde for deres ForsMmelse i at gjFre det Gode , hvortil Gud gav dem Anledning. la , forblindedc og hovmodige ere vi alle af Naturen , saa forblindede og hovmodige , at vi ikke ved vor cgcu Forstauds Lys eller ved vor egen Kraft kunne komme til en levende Erkjendelse af vore Synder , til en <b>sand</b> Idmygelse for Gud . Vi kunne nok af Samvittigheden drives til Erkjendelse af , at vi erc Syndcre og have Feil og men vi kunne ikke see , hvor store Syndere vi ere , kunne ikke forstaae , at vi have fortjcnt Guds Vrede og Straf , og at vi <b>selv</b> ikke formaae at frelse os derfrå , kunne ikke <b>selv</b> vcekkc os op af den aandclige Sikkerheds SMr , saa at vi af Hjertet hungre og torste efter Guds uforskyldte Naade . Nei ! det maa Guds Aaud lcere os ! han maa igjennem Loven tcende et nyt Lys i vor Sjcel ; men naar dette skeer , da lcere vi at see vort <b>Hjerte</b> og vor hele Natur som forgiftet af Synden , at sce vort hele Liv som syndbcsmit-

<h4>2261</h4>Bryllupsfest . Brudgommen er KongesMnen , som throner ved Faderens Haand i Himmelen , den Herre Jesus Christus, og Vruden er hans levende Menighed , som han har samlet og stadigcn samlcr af al Menneskens Sloeqt . De to elske hinanden i Aand og i <b>Sandhed</b> . Sec , hvilkcn hengivendc , altopoffrende , livsmeddelcnde og utroettelig <b>Kjoerlighed</b> ncrrer ikke Christus for sin Brud ! Ligesom Manden skal forladc sin Fader og <b>Moder</b> og blive fast hos sin <b>Hustru</b> ( Math . 19 , 5 ) , saaledes forlod Guds SM sin Himmel og den Fader , i bvis Samfund han levcde saligt fra Evighcd af , steg ned til den syndbesmittcde Jord , vandrede her som vor Broder i den dybeste Forncdrclsc og ~ hengav sig <b>sclv</b> " i Korsets marterfuldc for at ~ forhverve sig en Menighed med sit cgct Blod " ( Ap . Gjem . 20 , 28 ) . Og cfterat han har tiltjpbt sig sin Brud ved saa uendelig dyr en eftcrat han er indgangen til sin himmelske Faders Herlighcd , saa vedblivcr han at elste sin Menighed med densamme trofaste <b>Kjoerlighed</b> og bliver stcdsc hos den . Han ~ renser den ved Vandbadct i Ordet", fra Syndens Nreenhcd , bcercr taalmodigcn over med dens mange SkrFbeligheder , tilgivcr den dagligen alle dens Synder , velsigner den uafbrudt med sine rige Naadesgavcr , vaagcr utrcettcligen over den , styrker den med sin Seicrskraft , fMr ethvert af dens Lemmer paa de Veie , som han i sin Viisdom seer at voere de bedste , bcskjermer den mod alle dens Fiender , ~ for at han kan fremstillc sig <b>selv</b> Mcnighedcn herlig, uden Plet eller Rynke eller noget Saadant , men hellig og ustraffelig ( Eph . 5 , 26 — 27 ) , som en vidundcrlig deilig Brud ~ i hellig Prydelse " ( Salme 110 , 3 ) . Og Bruden , den sande Guds Mcniqhed — sa ! den er fattig og ussel i sig <b>selv</b> , den bcerer Naturens Vanart og dybe Fordcervelse i det , den synder dagligen af SkrFbelighed ; men dog har den givet Brudgommen sit <b>Hjerte</b> , dog elsker den ham , fordi han har elskct den fprst ; den elsker ham som den Elskeligste af Alt , hvad elskeligt den kan tcenke sig , som sin dyrcbareste Skat , mod hvilkcn den aglcr al Verdens Herlighed som Skarn ; den elsker ham , naar han glcedcr den med sine <b>Kjcerligheds</b> Gåver og lader sin Naades Sol lyse klart over den , og den elsker ham , naar han skjuler sit milde Aasyn for den og s > rer den ind i Trcengslerncs Ild ; thi den vccd , at ogsaa da er det hans trofaste <b>Kjoerlighed</b> , som raadcr , ogsaa da er det dens sande Vel , som han vil fremme ; den aabenbarer ham sin <b>Kjoerlighed</b> , idet den gjerne vil gloede ham ved oprigtig Troskab, idet den fornoegter sin egen Villie , strider imoo de Fiikke

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008090903023'target=_>Wallin, Johan Olof, 1848, Prædikener over de aarlige Søn- og Helligdages Evangelier</a><h4>2522</h4>Blomstret er faldet af ; Grcesset fortsrret . Blege ligge de spredte Blade ; tidlig Negn og sildig Regn kommer - — men den opfrisker det ei ; og Morgen- og Aftenstjerner tindre paa Himmelen — men i den ssrgende Sjel raader Nat . Den Gjenlevendes <b>Hjerte</b> er semderrevet ; derfor blsder det — er sammenpresset ; derfor sukker det . Vil Du ncegte det at blsde , da forbyder Du Mennesket at verre Menneske . Vil Du bebreide det sine Sukke , da glemmer Du , at Christus <b>selv</b> stod i Taarer ved Lazari Grav , og de , som stode omkring ham , sagde rsrte : Se , hvor han elskede ham ! . . . Nei ! Du maa ei forvende Guds Anordning, ei forandre vor Naturs Beffaffenhed , ei tilintergjsre vort <b>Hjertes</b> omme , hellige Krav . — Men naar , under frugteslss Strid mod overlegne Qvaler , — naar et eller andet trssteswst og bcevende menneffeligt Vcesen , allerede nedbsiet under Sorgens haardeste Slag eller forud anende , at de ville ramme - — forgjceves tilbageholder sin tause Misnsie over Guds underlige Raad , og ssrger med den stumme Smertes <b>Blik</b> mod det Hoie :  hvorfor blev da Guds Anordning saaledcs fastsat og vor Natur saaledes indrettet og vort <b>Hjerte</b> saaledes dannet " ? da bchsves et Svar , ikke et menneffeligt Svar fra Verden , men et guddommeligt Svar fra Helligdommen . Giv os det idag , o Trsstens og <b>Sandhedens</b> Aand ! lcer os at forstaae , hvorfor <b>Kjerligheden</b> og Sorgen her bleve saa troe Ledsagere ; tal til os om den hoie Viisdom , som gjorde det Blideste saa ncrr bestcrgtet med det Bittreste ; opklar dette Morke , saaat vi ogsaa deri maae ste en evig <b>Kjerlighed</b> , som vilde vort Vel , vor Foraedling , vor varige Seier over Tid og Jord , — saaat vi endog i den dybeste Nod med Tillid maae raabe til vor Gud og fornemme , at han er vor milde Fader og attid trygge bede , bsnhsrte bede vort Fader vor o . s . v.

<h4>2953</h4>O ! stor Uro er beredt hvert Menneskes Liv , naar det ikke leves anderledes , og ikke sluttes anderledes . Da maa vel spsrges : hvorfor er Lyset givet de Elen « dige , og Livet de bedrsvede Hjerter ? I sukkende og besvcrrede Vandringsmcrnd paa Jorden ! Da kommer Gjenvordigheden frem af Jorden , men Glceden skal borttages ; da bekymre I Eder med mange Ting , men have ingen Trsst i Eders Angest . Og alligevel staaer den <b>Sandhed</b> fast , at Himmelen er Her < rensThrone og Jorden hans Fodskammel , at han endnu udgyder sin Velsignelse ligesom en Bcrk , og at hans Trsstekilder ikke udtsrres . Men hvo har forhindret Eder fra at troe <b>Sandheden</b> ? Hindringen ligger i Eder <b>selv</b> . Eder fattes de tv ende Stykker , uden hvilke Intet ssges og Intet vindes med Hensyn til Eders Sjelefred , — de tvende Stykker , med hvilke fordum alle Guds Born paakaldte ham i al Nsd , og saae , at alle hans Veie vare Miskundhed og <b>Sandhed</b> , — de tvende Stykker , som i vort Evangelium lcerte en af Sor , gen rammet Mand og en haardt prsvct <b>Qvinde</b> — som fcelles Forbilleder for os Alle — med ydmyg Tilbedelse at soge , med taalmodig Fortrøstning forvente , med glad Er , kjendtlighed modtage Hjcelpen af den eneste Hjcrlper , Trssten as den hsieste Troster , Freden og Livet af den evige Fredsfyrste og Livskilde . Og — glemmer ei , Christne ! at disse Tvende ere Bon og Tro . Have I vel bedet til Gud ? Nei ! om I end af en gammel Scrdvane eller en uvilkaarlig Oieblikkets Bevcrgelse stundom holdt Eder noe r til ham med EdersMund og cerede hammedEders Lceber , saa var dog Eders <b>Hjerte</b> langt fra ham ; det vendte ei sin Hu fra Verden til Gud ; det overgav sig ei udeelt i hans milde Haand med Hengivenhedens fromme Suk : han er Herren , han gjere med mig , hvad ham behager ! Det hcevede sig ei til sin gode , fortrolige Fader med den inderlige Overbeviisnings Aand , som af dybb » ste Nsd stuer med klart <b>Blik</b> til Himlen og siger : Du vil , Du kan , Du gjer Alt , hvad Du har lovet , og som , <b>selv</b> om det for Vicblikkct eller for Tiden bliver

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111103033'target=_>Wallin, Johan Olof, 1854, Prædikener over de aarlige Søn- og Helligdages Evangelier</a><h4>543</h4>Folelsc af Liv : thi netop der bcgyndcr den frie Udsigt , den rette Besiddelse , det sande Liv . Derfor , naar I overlade Graven det tomme Kledebon af hvad der var Eder kjcerest , saa vide I , at <b>selv</b> dette Kjcercste er i Guds Haand og i god Forvaring ; det er alene frelst fra de Farer og Storme , blandt hvilke I endnu strcebe , men fra hvilke I ogsaa eengang skulle reddes paa den samme rolige Strandbrcd af den samme faderlige Haand . Bed denne Tanke forvandle Taarerne sin Bittcrhed til Behagelighed ; det tause Suk af Lcengsel eller Savn bliver Udodelighcdens Aandepust ; i hellige , unevnelige Folelscr udvider sig Eders Bryst for alt det Wdle og Skjonne , som foreviger Livet baadc i dets selvstcendige Bevidsthcd og dets uforgjengelige Besiddelse . De derimod , ak ! de betragte ogsaa med Alvor det ovstillede Maal , naar de ikke kunne skue forbi det , men det er et ncdslaaende og trosteslost Alvor . Der frcmbryder Morket : hvad stal itke ncdsvnke i det ? Der aabner sig Graven : hvad skal ikke ovsiu < ges af den ? . . . Der er noget Skrekkcligt i de Dine , som , efter et saadant Sporgsmaal , skue omkring sig ; det er forfcerdeligt at trykke mod sit <b>Hjerte</b> hvad man elsker og tcenker :  det er dog Stov , sammcnfaldende , forsvindcnde Stov ! " Kan vel , med en saadan Tanke , om det endog kuns var halv Tvivl , nogen <b>sand</b> <b>Kjerlighed</b> trives ? Og uden den er jo Livet blottet for Vcerd , som Doden er tom paa Trost . . . den alligcvel uundgaaelige Dod ! De Ulykkelige , lad dem kun vcere istand til at adsprcde sine morke Forestillinger og ikke see ind i Dybet af sin Haabloobcd , saa lenge deres Kraft og Lyst hcenge fast ved Verden og svoselscrttcs blandt dens forskjellige Formaal ; men Kraft og Lyst forgaac, de lange Skygger komme ncermere , de udvide og udbrede sig , jo mere Synskredsen sammentrenges , de samle sig stedse tykkere i Ellobelighcdens og Smertens Aftentimer , — og blive til Slutning staaendc uigjcnnemtrengclige over Sygesengen og Dodsleiet . Mork er da den Anelse , som svcevcr omkring det ode <b>Hjerte</b> , — bittert det <b>Blik</b> , som med stum Smerte falder paa de forbausede Omkringstaaende , skrekkelig den Afsked , som udtrykker sig i de sidste Bevegelser af Folelsen. Da fornemmes i <b>Sandhcd</b> , at Jesus er borte , som han ogsaa var borte under hele den foregaaendc Vandring ... og den , som staaer tilbagc , ak ! uden hans Sclskab , hvorhen ledes den ? hvorhen fores den ? hvor standser den ? — Bekymringvcekkende Sporgsmaal for Eder , som blive tilbagc , I Jesu Venner , som med al Eders Bon og Moie ikke have formaaet at fore Eders Ncermeste paa bedre Tanker

<h4>846</h4>Fred , dersom Verden , Forfccngeligheden , Forsommclserne , Lysterne skuloe fortcelle Alt , hvad de have tilegnet sig af din Tid , dine Krcefter , din Iver og dit <b>Hjerte</b> ? Men de behove ikke at sige det . Det er allerede skrevet , lige fra Begyndelsen , i den Bog , som stal oplades , naar Han kommer , som skal dsmme lorderZye med Retfcrrdighed . Da bcstaaer Du ikke for den hellige Dommer . . . Ak ! Du bestaaer ikke nu for den kjcerlighcdsrige Fader , som har gjort Dig saameget godt , optaget Dig i et saa hoit , saa herligt Bornestab , ladet Dig allerede her faae en saa rig Arv af Naade og Fred og Velsignelse og Saligbcd , alene pau det Vilkaar , at Du st ' ulde eljTe ham , fordi han elj ? ede Dig fsrst , med <b>Kjcerlighed</b> gje < re hans ' velbehagelighed og villigt lade hans gode Aand fore Dig i det jcevne Land . Men ak ! saa hurtigt forlod Du din fyrste Rjcrrlighed , din trofaste ledsager, og gik paa Mørkets ' veie , drog afsted med Verdens Born , for at sde dit Gods langt borte fra din Fader , i Ugudelighedens , Udsvccvelscns og de jordiske Bekymringers fremmede Land . Der bliver , seent eller tidligt , en svar Hunger i det samme Land , naar Sjcelen kjedes ved det meget ringe Brsd , — stor Sorg , naar Hjertet af Dybsens Nod seer tilbage og tcenker hvorfra det er faldet . Dm Hunger stilles ikke af jordisk Fode , den Sorg adspredes ikke af jordisk Lyst . Folg dem , hvorhen de ville fore Dig ; de ere Guds egen Aands Hendragen til Icsum . Ingen , siger Midleren <b>selv</b> , kan komme til mig , uden Faderen , som mig udsendte , faaer draget ham . Der kommer Ingen til Faderen , uden ved mig . Saalcdes talcde han , som er Neien , og <b>Sandheden</b> , og Livet . Underbare Kjcerlighcdsraad, forklaret i vor Naturs Inderste ! Guddommelighedens Hemmelighed, aabenbar for de Troendes <b>Die</b> ! I Guds Verden bevcege sig , ogsaa efter Faldet , ved hans milde Forsyn , uden Afladclse de aandclige Krcefter , udgaae fra hele Naturen de himmelske Nostcr , som , saalcenge Naadetiden varer , hvor som helst Synderen vanker om , banke med tcette Slag paa hans haarde <b>Hjerte</b> , raabe med tusinde Tunger i hans 2 ) re : vend om ! vend om ! Og fornemmer Du dette Slag , agter Du paa disse Naab , Synder ! saa er Tilbagevcien til Faderen endnu mulig for Dig , mcdens Dagen skrider frem , og fsrend Skyggen bliver stor . Men den Vei gaacr over Gethsemane og Golgatha . Mcd Bedrovclsens Ild og Bonnens Kamp maa dit <b>Hjerte</b> renses , at den rene og rette , om ogsaa i Angest fodte Bornctanke der kan faae Plads : Jeg vil staae op , og gaae til min Fader , og sige til ham : Fader ! jeg haver syndet mod Himmelen og for Dig , og er ikke lcengere vcrrd at kaldes din Sem . Med Angerens bittreste

<h4>1029</h4>dem er kommen til Verden , for dem bar levet , lidt og er dod , og som folger dem cndnu , under alle Provclser og Omskiftelser , med samme hulde , oplivende <b>Blik</b> , med hvilket han fordum saae ned paa dem , som stode ved Korset ftaa Golgatha . Han blev Menneske , ftaa det at vi skulde kjende og elske ham , som den menneskelige Mildhcds og Ncenhcds skjonneste Urbillcde , — paa det at han skulde verre det levende , herlige Ord , som over al Verden forklarer , at en uendelig <b>Kjcerlighed</b> er den Kraft , som giver hele Verden sit Udspring , sin Frelse og Bestaaen , — er det Baand , som sammenholder Himmel og Jord , og forener Jordens Born omkring den gode Hyrde , som scetter sit L. iv til for Faarene , og bevarer dem , som ham ere givne . Naar det hcdder om Moses , at Herren talede til ham , Ansigt mod Ansigt , ligesom en NTand taler med sin vicrste , saa er dette Noget , som Jesu Trofaste hvert Oieblik erfare i stille , opmcerksomme Hjerter . Derfor kunne de forsage Alt , og have dog nok , — vandre i Morket , og finde dog Veien , — provcs , og vcere undergivne i Taalmodighed , — lide , og vcere glade i Haabet . Uadskillelige fra den himmelske Ven ved sine Bonner og sin Fortrostning , hore de under al Tidens Uro , og dens Gjenvordigheder hans beroligende , ovmuntrendc Stemme , og vide stedse , baade hvad de have at gjore og vente . Fra deres synlige Omgang er han stilt , men ' , under sin himmelske Aands Veiledning , overdrog han dem at fortscette hans Kjoerlighedsvcerk ftaa Jorden . <b>Selv</b> ofthoict over Tidens Skjcebner og Dodeliges Deeltagelse , anviser han dem de Gjenstande, som de have at vogte , indtil han kommer og gransker Troskabsproven. Ingen , som vil have Deel i ham , maa da tilslutte sit <b>Hjerte</b> , naar han taler . See , det er din <b>Moder</b> , din Fader , giv dem Trost og Understottelse , medcns Du haver dem ; jeg stal derfor velsigne Dig . See , det er din Sem , din Datter , fob mine Lam , at de ikke forvildes eller kommer bort ; jeg vil bcere dem ftaa mine Arme til min Fadcrs Huus . See din Broder , ja , din Broder , i hvert Medmeneske , — viis tilrette , hjcelp , husval , forson , hvor Du kan . Du kan meget , naar Du vil det af Hjertet , — og saa meget , son , Du har gjort een af disse mine mindste Brsdre , har Du gjort for mig . Saaledcs taler han , og hvo det horer og gjor derefter , han giver det rette Troskabsvant , og haver det rttte Tegn , ftaa hvilket Herren vil kjende Sine , og bliver i ham , og skilles ikke hverken af K.iv , ei heller af Dsd fra hans Rjcrrlighed . O ! hvem skulde ikke saaledes ville stille sig ved Jesu Kors ? Og hvor skulde ellers et Menneskcbrpst beroliges og bctryggcs ? — O , min Frelser !

<h4>1091</h4>ikke Jesus , cfter sin Dod , en anden ; alene forklaret , klccdt i Uforgjccngelighed, straalende i himmelsk Majestcrt , syntes han ikke lccngere at tilhore Jorden . Men det samme <b>Hjerte</b> stog i hans Barm , den samme <b>Kjcerlighcd</b> og Fred skncde ud af hans <b>Die</b> , den samme Naadc og <b>Sandhed</b> stroinmede fra hans Lceber , den samme trofaste , varme Haand raktcs den , som modte eller fnlgte ham . Fol dit Haab , Christen ! naar Du overgivcr dit Dyrebareste til Graven , eller <b>selv</b> stal nedstige i dens morke Gab . Paa den Vei , hvor Jesus har gaact , kan Du hverken tabe Dig eller Dine . Den Retfcrrdiges Sti er sonl et skinnende L. ys , der fremgaaer og lyser indtil he » i Dag . Men her fremgaacr ikke den fulde Dag : Der ere Retftrrdige , hvilke det rammes efter de Ugudeliges Gjerning ; her synes Livets Lodder sjelden afveicde efter den Frommes Onster og Forventning ; forunderligt , uoverskuelig ! reise sig Hccndelserne og Tingene , under Sloret af en tung Tilskikkelse skjuler sig mangen Gang den evigt Mildes Swrelse , saa at baade vore Folclser oprores og vore Tanker forvildes ; Naturens los ^ revne Krceftcr true baade den Skyldige og Uskyldige ; Ulvkkcrne , Smerterne, Skrobelighederne hcerje uden Hinder af Netskaffcnheden , uden sErefrygt for Dyden ; Dodsengelen gaaer ingen Dor forbi og der , hvor den kommer , hoster den , uden Hensyn til Styrke eller Svaghed, Knurren eller Hengivenhed , Glcede eller Klage . Derfor kommer en Gjengjcrldelscns og Forklaringens Dag ; Gud har nedlagt dette Lofte baade i sit Ord og i vore Hjerter . Derfor maa vi Alle gaae over til Aandernes Verden med Mindet om hvad vi have haft , crfaret og gjenncmgaaet , med levende Bcvidsthcd om hvad vi have handlet , tcenkt og vccret . Og naar — efter Dodens korte Afmagt og Slummer, dette Minde ovvaagner med fornyet Styrke , denne Bcvidsthcd vender tilbage i fuldkommen Klarhcd , hvorhen , o hvorhen stal vel Sjcclcns lccngtende Vlikke forst vende sig , dersom det ikke skal vcere efter dem , som allerede i Tiden have virret vor kjcere Tilstugt , vor forste Glcede , vort kjcercste Selstad ? Hvad vilde en Himmel vcere uden din gode Wgtefcclle , Du eenslige Wgtemage ? . . . uden dit elskende Barn , Du forgende <b>Moder</b> ? . . . uden din trofaste Ven , Du eftcrladte Ven ? ... I fole det , omme , rene Sjcele ! I skulle leve : hvorledes kunne I det , uden at elske ? I stulle blive evigt lykkelige : hvorledes kunne I det , under et evigt Savn af Gjenstanden for Eders Kjcrrlighcd ? I skulle gjenkjende Eder <b>selv</b> : hvorledes kunne I det , uden at igjenkjende dem , som udgjorde en Bestanddeel af Eders egen Tilvcrrelsc? — I skulle soge dem : det siger Eders <b>Hjerte</b> . I skulle finde dem : det siger Guds Godhed . I trofaste Jesu Disciple ! Lders

<h4>1121</h4>Men dersom alle vore ceblere Egenskaber i Fellesstab bor medvirke til denne glade Overbcviisning , denne klare Indsigt , dersom ikke en eneste af de Evner , som udmerke os fremfor Dyrene , kan ustraffet kvceles eller endog kun behandles med Ligegyldighed , naar det gjceldcr at soge den rolige Overeensstemmelse med Gud , Verden og os <b>selv</b> , som alene fortjener Navn af Religion : saa er det dog forst og fremst Hjertet , som , naar en <b>Sandhedens</b> Straale falder ned fra det Hoie , maa paa vor Side vcere Grunden , Midlet og Vilkaaret for dens Modtagelse . Ligesom den rene Kilde , i hvilken Himmelen skjonnest afspeiler sig , saaledes opfatter et reent <b>Hjerte</b> baade meest trosteligt og tydeligst Guds Riges Hemmelighed . Grumses eller fortorkes denne klare Bolge , hvad blive da Viisdom og Forstand uden en puffende Rilde og Nande , der ikke ere besiandige ? Hvad blive da alle Lerdommens Skatte , alle Geniets Opdagelscr , alle Kulturens Bchageligheder , uden den Gjerriges unyttige Forraad , ved hvilke han <b>selv</b> hungrer ihjel ? Bevar derfor , o Christen ! bevar dit <b>Hjerte</b> frem for alt det , der forvares , thi L. ivet udgaaer af det . - den sande Gudfrygtigheds Liv , som i Eenfoldighed og <b>Kjerlighcd</b> strider fremad til Lys og Fuldkommenhed . Vor Frelser priser ikke den : salige , som i vidtstrakte Kundskaber , mangfoldig Kunst og skarp sindigt Vid ovcrtrceffe sine Ligemcend ; han siger ikke , at de fremfor Andre skulle i aandelige Ting vide meest og skue klarest ; mi , han siger : Salige ere de Rene af Hjertet ; thi de stulle see Gud . Og kunne de ikke endnn i Alt opdage den uendelige Viisdoms Raad , er der endnu i mange Dele et uigjcnncmtrengcligt Slor , som forhindrer deres Vlik fra at skue den himmelske <b>Sandhed</b> , saa siger han til deres Trost : Salige ere de , som ikke see , og dog troe . Hvad vi her see , mine Christne ! ere alene ydre Skin , tomme Skygger , foranderlige , kolde , forsvindcnde Skikkelser . Standse vi herved , som Saamange gjore , og betragte det som det ene Virkelige , indskrcenket og usel bliver da vor Viisdom , den er ikke tilstrekkelig med hele sin antagne Ufeilbarhcd til at forklare Menneskelivet , Verdcnsstyrelsen og vor tilkommende Lod . Men det Virkelige , det Sande og Vesentlige ligger hoicre end al jordisk Hovmod , dybere end al verdslig Dybsindighcd. Vi kunne ikke see det , thi det Aandelige er usynligt ; vi kunne ikke omfatte det , thi det Guddommelige er uendelig ! ; vore <b>Oine</b> ere betagne , og maa vcere det , indtil Dekket falder . Men foler vort <b>Hjerte</b> sine hoicste og helligste Fornodcnheder , brcender det i hellig <b>Kjcerlighed</b> for Alt , som oplofter og foredler Menneskenaturen ; hever det sig higende over en forfengelig Verdens baade Viisdom og Daarstab, op til det evigt Gode , Sande , Skjonne og Hellige , for hvilket

<h4>1191</h4>og han <b>selv</b> blev stedse mere skjult for dets til Jorden scenkcde <b>Blik</b> ; hans Stemme i Hjertet og Naturen blev uforstaaet og hans hellige Nccrvcerelse forglemt . Dette er Gangen af vor Slcegts Skja ' bne , forsaavidt som den har vceret afhcengig af sin egen Indsigt og Kraft . Dersom ikke Aabenbarelsen satte dette Forhold udenfor al Tvivl , saa skulde Eftertanken og Erfaringen allerede bevidne det samme . Men nu ere alle baade ydre og indre Vidnesbyrd enige deri , at Mennesket ikke er udgaaet af Skaberens Haand , saalcdes som det er , at det er dette ved sin egen Skyld og at det , med alle sine Bestrebelser ikke er blevet hvad det burde . Vel har det gjort sig Umage ; det har ikke kunnet hvile ; det har , <b>selv</b> uden at gjore sig tydelig Neoe derfor , savnet og sogt det Tabte paa tusinde Veie . Men hvor og hvorledes har det fundet det ? See , Msrke skjulte Jorden , og Dunkelhed Folkene ! Til stumme Afguder gik de , til Vildfarelser og Forbrydelser forfaldt de , af Splid og Vold bleve de sonderstidte . De simpleste , forste Grunddegreber om <b>Sandhed</b> , Ret og Sceder vare fordunklcde eller kvalte . <b>Selv</b> den formeente Nodvendighed af at formilde fortorncde Guddomsmagter, frembragt ved Folclsen af en brodebetyngct Samvittighed og det undertrykte Behov af en Renselse for Sjcelen , yttrcde sig enten med Overtro eller med Grumhcd , med Uretfcerdighed mod Gud eller mod Menneskene . <b>Selv</b> de Vise og Bedre — hvad vidstc de ? Hvad vilde de ? De betragtede med Medynk Hobens Blindhed og Daarskab , men havdc <b>selv</b> intet klarere Lys og ingen sikkrere Trost . Hvilke taabelige Meninger om Verocns Ophav og Styrelse ! Hvilke vaklende eller modstridige Forestillinger om vore Pligtcr og indbyrdes Forholde ! Hvilken Tvivl , hvilken Uvished , hvilken Nat over Menneskchedens Maal , V ^ rrd og hoiere Forbaabninger ! . . , Tcenk herover , Du som har modtagct rene og tilfredsstillende Begreber i saa scerdeles vigtige Sager ! Tccnk herover , og sporg siden din Fornuft og dit <b>Hjerte</b> , om Skabcren kunde lade sit cedleste Vcrrk paa Jorden blive tilintetgjort og den synlige Skabclses Krone falde ned fra dens Hoved i Stov og Vancere ? Om det ikke var den hoie Viisdom vcerdigt , at sende den Hyrde , som alene kunde opsoge de tabte Faar ? ... Og han kom , da hans Tid var der : ikke med Hcermagt og Storm , for at overraske , tvinge og underkue , men med <b>Sandhcd</b> , <b>Kjcerlighed</b> og Fred , for at oplyse , bevcege og tilrettevise : ikke i guddommelig Majestcet , ikke i jordisk Kongedragt , men i en tjeners Skikkelse , i Selskab med Lidcl ' ' erne , Fattigdommen , Foragten og Simpclheden ; thi det Guddommelige skulde ikke seire ved jordisk Krafi , og det Fortabtc skulde ikke komme tilbage ved Tvang , men ved Frihcd . Saaledes udgik ogsaa hans Apostle , simple , fattige

<h4>1219</h4>Christne ! kjende vi nu vor gode Hyrde , vi , som her saa mangen Gang tabes , forvildes , saarcs og formaae Intet ? . . . Naar han opsoger det Tabte , forer det Forvildede tilbage , forbinder det Sonderbrudte og styrker det Svage , — da kjende vi ham og hore hans Nest , og vi have ikke megen Rummer og et beklemt <b>Hjerte</b> , thi da vandre vi , fulgte af hans <b>Die</b> , glade ved hans Gaver , hvorhen han <b>selv</b> er gaaet forud for os eller der han <b>selv</b> sender os . Da jeg udsendte < kder , spurgte han Disciplene , haver Noget fattes Eder ? Da sagde de : Intet . — De havde jo nok under alle Mangler , Lys i alt Morke , Trost i hver Sorg og Styrke i alle Gjenvordighedcr ! . . . Hvad Andet behover Du , Guds hjemvandrcnde Barn ! Hvad mere begjccrcr Du ? — Har Du sundet det , saa behold Du , hvad dit er og siig med frcidigt Mod : Herren er min Hyrde , mig skal ikke fattes ! Han lader mig ligge i skjsnne Grcrsgange ; han leder mig til de stille rindende ' vande : vaa <b>Sandhedens</b> og Naadens evigt blomstrende Grcesgange ncerer han min Tro og min <b>Kjcerlighed</b> ; af Husvalelsens aldrig udtorrende Kilde opliver han min Kraft og fornyer daglig mit Haab . Han omvender min Sjcrl ; han fsrer mig paa 2vetfcerdigheds Neie for sit Navns Skyld . Naar jeg end skal vandre i Dsdens Skygges Dal , vil jeg ikke frygte for Ondt , thi Du er med mig ! Godt og Miskundhed stal efterjage mig alle mine Livs Dage , og jeg skal blive i Herrens Huus en lang ja ! trofast her i den stille Faarcsti og saligt engang i det himmelske Faderhuus! Amen .

<h4>1555</h4>Elskede ! ikke Jorden , men Himmelen indeslutter dem ; forstaacr dct himmelske Sprog , som taler til Eders Hjerter fra en Ven eller LEgtefcelle , en Broder eller Soster , en Fader eller <b>Moder</b> i de Levendes Land : I have da vel nu Bedrøvelse , men jeg vil see Eder igjen og Eders <b>Hjerte</b> skal glcrdes , og Ingen tåger Eders Glcrde fra Eder . Og Du , Lazarus ! som i Nod og Elendighed forsmcegter ved den übarmhjertige Fraadsers Dor ! hav Taalmodighed , lad din Sjcel forccdlcs af Ulykken , ligcsom Guldet luttres i Ilden , og tcenk siden , ( hvad Du kun da kan tcenke ) , at Gud haver udvalgt den » , son , ere fattige i denne verden , til at vorde rige i T ^ roen og Arvinger til Aiget , O ! vi Alle , som her , fulgte af Ekrobcligheder , saarede af Lidelsen , ncdsynkcnde under Byrdcrne , nogle Oieblikke sukke i en henfaldende Hytte , vi ere altid frimodige , om vi end bsr at indgaae i Guds Rige med mange Trengsler , thi vi vide , at dersom vor Hyttes jordiske Huus nedbrydes , saa have vi en Bygning af Gud , et Huus , som ikke er gjort med Hcender , evigt i Himlene . Gnd raader baade for det Liv , som nu er , og det tilkommende og Gudsfrygt haver Forjettelse for begge . Vor Bane er uendelig ; lader ikke vore Fodder snuble eller trccttes ved disse forste Skridt ; lad ikke vor Tro forstyrres , men heller befestes af den begyndcnde Vandrings Gjenvordigheder ; lader os ikke bedomme , hvad vi endnu ikke kunne overscc , men rolige forvente , hvad den hoieste Viisdom og <b>Kjcerlighed</b> har opbevaret for rene Hjerter . Lader os vende <b>Oiet</b> med Trost fra det Sorgclige , Dunkle , Uforklarlige , som her saa ofte omgiver og ncdslaaer os : det er ikke alene en Overgang , det har en Indstydelse vaa lysere Udviklingcr , dct hensigtcr til hoiere Formaal , det staaer i Sammenhang med evige Skjebner . Saaledcs lerte han , som bragte Liv og Uforkrcrnkelighed for Lyset ved Evangelium , han , som ikke alene under hele sin jordiske Vandring kun virkede for Himmelens Formaal og i Doden befalede sin Aand i Faderens Hender , men ogsaa opstod af Graven , for at lceggc for Dagen , hvorledes den samme Almagt , som har frembragt Menneskelivet , ogsaa kan opretholde og fornye det ; og Enhver , som har Sands for Menneskenaturens Ophoielse , det Godes uforgjengelige Verd og den evige Orrens Nodvendighcd , omfatter med Hjertets fulde Overtydning denne hoie , gl ^ derige , Alt forklarende <b>Sandhed</b> .

<h4>1624</h4>dog , at han er Middclvunktet , til hvilket alt Andet bor henforcs , omkring hvilket de ovrige Ting og Skabninger maa dreie og bevege sig , ligesoin lavere Verdenslegcmer omkring sin Sol ; ellers sinder han Verdens Gang urigtig , uregelmessig og forsoger gjerne , saavidt nmligt , at rette den efter sit Vegreb og sit Behov . Alt Andet er kun et Middel for hans Hensigter . Saalcenge det er tjenligt for Dicmedct , har det forholdsviis noget Vcerd ; naar det har gjort sin Tjeneste , maa det bortkastes , ligcsom ' Skallet eftcr den udpressede Frugt . Ingen Agtelse for Ligcmends Net eller billige Forventning , intet Baand af Taknemlighed , af samvittighedsfuld Asrcsfolclse , af Tillid til givet Loftc besverer ham , naar Tilfcrldet tilbyder en Fordeel , kildrer Forfcengeligheden eller opfordrer til Nydelse . Han standses da ikke af nogen Bctcenkelighed , end mindre af nogen indre Strid , og kan ikke engang kaldes fnstet , thi han er strax vunden og altid beredt . Hvad der er hclligt og herligt for Menneskeheden , om han endog i store Ord forer , det paa sin Tunge , sinder alligevel ikke nogen modsvarende <b>sand</b> og varm Folclse i Barmen . Religionen , Dyden , Fedrclandskjcrrlighcden, Videnskaberne , de skjonne Kunster : Alt maa tjene til haandgribelig Nytte , og Nytten vere hans , ellers er den ingen , og ingen Glade derover kan sinde Plads i hans snevre , jordiske Sind . — Derimod er det himmelske Sinds Glede reen i sin Grund . Den , som har lcert at henforc Alt , ikke til sig <b>selv</b> og sit Stovs forgjcengelige Vesen , men til Gud og sin hoiere Bestemmelse , har allerede derved stillet sig paa det Standpunkt , hvorfra han ikke alene med et afSandselighcdcns Blendverk uforvildet <b>Die</b> skiller det Sande fra det Tilsyneladende, men ogsaa med et af Begjceringcrne übestormet <b>Hjerte</b> nyder det Gode , som den himmelske Fader allerede her i Tiden forunder sine Born . O ! han sinder det og gjcnkjendcr det , hvorsomhelst hans Vei end falder . <b>Selv</b> der , hvor almindelige Sjele ikke opdagc det , frembydcr det sig af sig <b>selv</b> for ham , thi hans Sind er stedse aabent , stedsc stemt for dets herlige , üblandede Indtryk . Tet <b>moder</b> ham i Religionen , hvis hoie Lerdomme , hellige Bud og salige Loftcr udgjore hans Tilstuost , Veiledning og Trost under alle Provelser og Omskiftelser , — i Naturen , som overalt omslutter , vederkveger og fornoicr ham med sin Nigdom , sin Storhcd og sin Skjonhed , — i den villige Udovclse af Pligtcrne , som hvert Oicblik opfordrcr til Virksomhcd og ligclcdes hvert Dicblik belonner sig <b>selv</b> , — i Samlivets Behag , som , under gjcnsidigt Udbytte af Tanker og Folclscr , paa sannnc Tiv opmuntrer og foredler , — i Huslighcdcns Ro , son : , under indbyrdes Vclvillie og Fortrolighed , fjerner Tvangen , gjor Moisommelighcden

<h4>1828</h4>sikkre Tone , som det antager under visse Olnstendigheder , stedsc svag , usikker , tvivlraadig og ulykkelig — og det bliver det stedse mere , jo mere upaalidelige og sirobeligc det sinder sine sedvanlige Tilstugter , jo hyppigere det <b>moder</b> de belcerende Sorger og jo nermcre det tommer til de übehagelige , men uundvigclige Vesindelsens Aar , da den legende Fantasies eller den selvbedragne Spidssindigheds Synsforvildelse bortdunster og i skrekkelig <b>Sandhed</b> fremstiller alt det Tomme og Trosteslose i dets indbildte Viisdom . Det hender ikke sjeldent , at det da gaacr over til en modsat Iderlighed , en Vantro , en Treldomsfrygt , som hverken fordres « ller billiges i Guds Ord , om det ikke ogsaa saa ofte , — for ikke at udscette sig for Andres Omdomme , — med en Forherdelse , som forkastes af det ncdslagne Sind , fortsetter med i det Lengste at ville synes den Samme ; indtil det i et uventet Tiebilk med Forferdelse seer den aabncdc Grav og da i sin Fortvivlelse vaakalder Religionens for det meestc for sildige Husvalelse . Laver os prise Skaberen, mine Tilhorere ! fordi han gav os Fornuft og en fri Villie . Men denne Fornuft kan , i vor nerverendc Stilling , ikke alene tilrcttevise os i vor Vandrings Morke , den lader vort <b>Hjertes</b> ommcste Sporgsmaal übesvarede og staaer raadvild og ukyndig , hvor vi allermeest tiltrenge en fast Fortrostning og en rolig Ovcrbeviisning . Denne vor Villie er , ved Sandsclighedens tunge Overvegt , for let , for svag , til af egen Kraft at modstaae Fordervelsen , at boies til det Sande og Gode og stride fremad i deres rctsindige Udovelse . Vi kjende det af vor egen Erfaring , og finde det bestyrket ved et <b>Blik</b> paa de Folkeslag, som , übekjendte med det evige Livsens Ord , endnu sidde i Msrke og Dsdens Skygge . Forftnet kjender sin Tid , da ogsaa de stulle bestraales af Evangeliets Klarhed ; men lader os ikke egensindigt og utakncmligt tillukkc Dinene for det Lys , hvoraf vi <b>selv</b> ere i Besiddelse . Hvad der har veret meest bclligt og af storste Vigtighed for Millioner Mcnnrstcr , bvad der har udgjort de bedste , de edleste Dodeliges Tilfiugt, Veiledning , Stotte , bor i det mindstc fortjene enhver Tenkendes omme Opmerksomhed . Der gives ' , mine Venner , i denne Henseende virkelig en T ^ roespligt , som den meest egenkjerlige Fornuft ikke bor kunne forkaste ; thi det er vel en Pligt , naar Folelsen af vore Feiltagelser , vore Mangler , vor urolige , haablose og übestemte Streben er given , at soge overalt , hvad der kan veilede os , styrke os , berolige os . Vi sogte det i Verden , vi sogte baade hos dens Vise og hos dens Daarer : fandt vi da , hvad vi sogtc ? Vandrede vi tryggere , blcve vi bedre , folte vi os mere tilfredsstillede ? Ak ! det er ikke Verden, ikke Sandseligheden eller engang Begrebet , som kan sige : Rommer

<h4>1877</h4>Indflydclscr , men ved en fottsat omhyggelig Benyttelse af de Midler , Gud har forlenet Dig — , dc Krcrfter han har givet , til Lys og Forcedling. Har Du denne ydmyge Selvtundskab , denne oprigtige Anger , denne levende Tro , denne redelige , vedholdende Bestrebelse for at bedre Dig , saa kan Du med stille Forhaabning sce op til en barmhjertig Gud , som er sterre end dit <b>Hjerte</b> , <b>selv</b> om det fordsmmer Dig . Du skal da med Fred og Glcrde erfare , hvorledes det er et fast Ord og vel vcrrd at annamme , at Christus er - kommen til Verden , for at frelse Syndere , og hvorledes saa Mange , som ham annamme , hore hans Ord , folge hans Fodspor og tilegne sig hans Forsoning , dem giver han Magt at blive Guds B ^ rn . Du skal da forstaae , hvad der har forvolvet al din Urolighed , din Ulyksaligheo , og sidenefter undftpe det ; Du skal ogsaa forstaae Guds viaade og Gave , og finde deri din eneste Tilstugt , din hoieste Trost , din cedleste Kraft : thi Syndens Sold er D ^ den , men Guds i ^ aadegave er et evigt Liv i Chrisio Jesu . Med hvilket andet <b>Blik</b> betragter Du da Verden og Tiden ! Med hvilket helligt Mod <b>moder</b> Du da Fristelsen og Provelserne ! Med hvilken uforstyrret No modtagcr Du da baade de onde og de gode Dage ! Med hvilket kjcerlighedsfuldt Sind omfatter Du da dine Medmennesker , som samme Faders Born ! Med hvilken inderlig Iver passer Du da alle dine Pligter , som Hverv fra den evige Godhed ! Med hvilken lus Forhaabning skuer Du frem mod det salige , for hvert Oieblik sig ncermende Maal som til dit rette Fcedrenehuus ! Du Guds gjenfundnc Barn , over hvilket hans Engle allerede glcede sig ! Du nye Mennene , son » er st ' abt efter Gud i <b>Sandhedens</b> Retfcerdighed og Hellighed ! Du var tabt , men er igjenfunden . Duvar dsd , og er atter dleven levende . Tak ham , som opvalte Dig og gav Dig Naade til , opstandcn af Syndens Dodssovn , at vandre i et nyt Levnet. Du er allerede indgangen i Guds Naaderige : Bliv der ! saa har Doden ingen Magt over Dig ; saa er Du ogsaa allerede indvict til et saligt Medlem af Herlighedens Rige . I Vishcden om dine Synders Forladelse , i Folelscn af Guds Velbehag , i Samvittighcdens uforstyrrede Fred , i Hjertets fromme , trygge Forventning om det tilkommcndc Gode , haver Du , allerede her , et evigt Liv . Hold fast ved det , Du haver , at Ingen skal tåge din Rrone : den fortjener , at Du kjcemper for den , vaager over den , beder om den . Saa strid da , uden at blive treet , indtil Du for altid har stridt den gode Strid ! Vaag , uden at indslumre , indtil Du engang faacr sove i Ro , for atter at opstaae til evig Glcedc ! Saa bced , uden nogen Aftadelse , indtil

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009073000103'target=_>Wexels, W.A., 1843, Forklaring over Det nye Testamentes Skrifter</a><h4>1173</h4>og Liv og <b>Sanbhed</b> ? hvorerdenosnceri sin Fylde ? ncrr , saa denne Fylde bliver vor , saa vi kun behsve at udstrcrkke vort Hjerte, for at gribe den , for at kunne sige : jeg eier al den Lydighet ) , al den <b>Kjcrrlighed</b> , al den <b>Sandhed</b> , Loven krcrver ! — hvor er den os saaledes ncer ? I Chr i sto Jesu . Den , der spsrger ester Loven for at opfylde den , den , som altsaa " hungrer og tsrster ester Retfcrrdighed , " den spsrger da i Grundcn ester Christus , om han end ikke <b>selv</b> veed det ; men , er christelig Tro tilstede , da hedder det : jeg spsrger ikke lcenger : hvem bringer mig Lovens Fylde , den Retfcrrdighed , Loven kraver ? thi dette vilde verre det Samme som at spsrge : hvem bringer mig Christus fra Himmelen eller fra de Dsdes Rige ? og han er jo kommen, han er jo opstanden fra de Dsde , og i ham min Retfcerdighed, i ham Lovens Fylde , der er mig ncer , som det Troens Ord , jeg hjertelig bekjcnder . Hvad altsaa Moses end har tcenkt paa , da han talede de Ord , Apostelen her har for <b>Oie</b> , og hvad end den ncrrmeste historiske Betydning af dem monne vcere , saa er det vist i alle Maader , baade at der klarliqen i det Capitel hos Moses , hvortil Ordene hore , spaaes om Metzsice Tider , da " Herren skulde omffjcrre Israels <b>Hjerte</b> til at elske Herren deres Gud af ganske <b>Hjerte</b> og af ganske Sjel " ( 5 M05.30 , 6. ; sml . Ez . 36 , 26. 27. 1 er.31 , 31 - 33. ) , og at den prophetiske Aand aldrig taler og heller aldrig , uden at modsige sig <b>selv</b> , kunde tale om Lovens Opfy ldelse , uden derved at pege paa Chr i stum , den Eneste , der har opfyldt Guds Lov , har opfyldt den for os til Rctfcrrdighed , og opfylder den i ilden ios til Helliggjsrelse . Vil man sige : la , det er da vel ogsaa denne sidste Opfyldelse , den i vore Hjerter til HelUggjsrelse, som Mose Ord egentligen maae henfsres til , — saa kan det verre ganske rigtigt , kun at vi mindes , at uden Retfcerdiggjsrelse er ingen Helliggjsrelse ; kun ved at annamme og tilegne os Christus for os til Retfcrrdighed , faae vi det <b>Hjerte</b> , der kan elske Gud i Aand og i <b>Sandhed</b> . Christus for os og Christus i os , den tilregnede og den i Sjelen indgydte Retfcrrdighed , maae visselig vel adskilles og kan ikke sammenblandes uden at forvirre det hele christelige Liv ; men paa den anden Side kan Talen ligesaalidt verre om Christi eller

<h4>1287</h4>" la , men Grenene bleve jo afbrudte , for at jeg stulde blive indpodet ; ved Isdernes Fald stulde jo Saliggjsrelsen komme til os Hedninger ! " — og kunde derved ville sige saa meget som saa : " ogsaa vi maatte da vel altsaa have en Vaerdighed for Gud og vort Kald til crdle Oliegrene ogsaa ligge i hans Bestemmelse ; ja , siden han forherdede sit eget Folk , for at komme til at frelse os , saa synes det jo endogsaa , som om vi stode end hsiere i hans Dine , som vare vi de Kjcere , for hvis Skyld han hengiver hvad han ellers kalder sit Kjcrreste . " Apostelen indrsmmer, at det er ganske <b>sandt</b> , at Gud ved Israels Forherdelse havde Hedningernes Frelse for <b>Oie</b> ; men han gjor tillige opmcerksom paa , at , ligesom der hos Isderne laae en indre Grund til Forherdelsen , nemlig Vantro ( 9,32 .33 . ) , saa deres midlertidige Forkastelse ingenlunde var en vilkaarlig Handling af Gud , grundet paa en ringere <b>Kjerlighed</b> til dem eller ringere Villie til deres Saliggjsrelse , saaledes var det kun ved det ydmyge <b>Hjertes</b> Tro paa den frie og uforskyldte Naade i Christo , at Hedningen stod i.Samfundet med Herren , stod som et levende , saligt Lem af hans Memghed , og ingenlunde fordi Hedningen i sig <b>selv</b> havde nogetsomhelst Fortrin for Joden eller var elstet fremfor denne af Gud . Istedetfor at hovmode sig , fiulde han derfor meget mere see sig vel for , at han ikke — hvad netop , naar han gav Hovmod Ncering , vilde stee — <b>selv</b> faldt ud af Naadens Stand , og , istedetfor at domme Andre , stulde han " arbeide paa sin egen Saliggjsrelse med Frygt og Bevelse " for at stulle miste denne ( Phil . 2 , 12. ) .

<h4>1376</h4>V . 17. Apostelen vender igjen sit <b>Blik</b> til Modstanderne og Forfolgerne ; men , ligesom han i V . 15 og 16. ikke har ladet disse ude af Syne , men ogsaa med Hensyn til dem givet de i disse Vers indeholdte Formaninger , saaledes blev ogsaa de Paamindelser , han nu i de folgende Vers af Capitlet giver , at anvende af de Troende , forsaavidt de indenfor selve Menighedens Grendser fandt Modstandere og Fiender i Saadanne , der vel bekjendte Christi Navn , men beherstedes endnu af Fordomme og Lidenstaber , og havde ei givet Naaden Raaderum i Hjertet , ei boiet dette under <b>Sandheden</b> og <b>Kjcrrligheden</b> . " Betaler ikke Nogen Ondt for Ondt ! " Her ( sml . B . 19. ) bestrides en af de mcegtigste Tilboieligheder , der findes i det fordervede <b>Hjerte</b> , en Tilboielighed , der , <b>selv</b> om den lcerer at dempe sine groveste Udbrud , dog altid veed paa en finere Maade at staffe sig Luft og dertilmed under stjemne Navne at besmykke sin urene , af

<h4>3193</h4>havde at tilgive , faa vilde han dog , ligesom han havde deelt Kramkelsen , der var vederfaret Menigheden , ogsaa gjerne dele Tilgivelsen , og de kunde verre saa meget des vissere paa , her at faae ham med sig , som han allerede , " dersom han havde tilgivet Noget " d . e . hvis overhovedet Talen fra hans Side kunde vare om Tilgivelse ( ste nys ovenfor ) , havde " for deres Skyld , " med Hensyn til dem , til deres Bedste , for at foregaae dem med Exempel , tilgivet , " for Christi Aasyn " ( vil vel ikke alene sige : oprigtigen , reent og <b>sandt</b> , eller , som Undre vil , med Christus og hans Sag for <b>Oie</b> , men tilkjendegiver tillige hvad der i det Forborgne , alene for Herrens Sine , var steet og nu aabenbarer sig ) . Han vedbliver : " paa det vi ( det maatte gjere et godt Indtryk , at han indbefattede sig <b>selv</b> med i Faren ) ikke stulle besncrres af Satan , " nemlig ved overdreven Stramghed , Mangel paa Forsonlighed , paa den rette Kjcrrlighed; " thi hans Anslag o . s . v. " d . e . vi kjende godt af egen Erfaring den onde Fiendes Traedsthed og Planer mod Guds Rige ; vi vide , hvorlunde han , naar han ikke kan holde os fast i sine Snarer paa den ene Side ( her : Ligegyldighed og Mangel paa gudelig Nidkjcrrhed for Herren og for Menighedens Reenhed ) , soger at naae sin Hensigt paa den anden Side ( her : en uaegte Nidkjaerhed , der forlader <b>Kjcrrligheden</b> , og bevarer personligt Nag under Skin af gudelig Iver ) . Upostelen tilsigter ogsaa de ulykkelige Felger , som Mangel paa Tilgivelse vilde have paa hiin Synder <b>selv</b> , og som jo Satan ( Modstanderen af al Guds Villie og Menneflenes Salighed) gjerne saae . Sml . V . 7 og Forkl . dertil . W . 12. 13. Apostelen vender tilbage til Fremstillingen af sit <b>Hjertes</b> Forhold til den korinthiste Menighed og sin Stilling overhovedet som Lcerer , navnlig som Korinthiernes . Nærmest kan Verset forbindes med V . 4. Det Mellomliggende var ncrrmest foranlediget ved Ordene : " ikke for at I stulle bedreves," i sidstncrvnte Vers . " Da jeg kom til Troas , " nemlig paa min Reise ftaEphesus ; ste Indledningen . — Troas , Navnet paa Landstabet i det Nordlige af Mysien , hvori det gamle Troja laae , er her at forstaae om Staden Troas i Nærheden af sidstncrvnte . See Forkl . Up . G . 16 , 8. — " En

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092303042'target=_>Rosenius, C.O., 1868, Hemmeligheder i Lov og Evangelium</a><h4>77</h4>bittre Tab og dybe Sorger , langvarig Fattigdom og trvkkcndc Gjeld , vedholdende Syndefristclser , ynkelig Svaghed og virkelige Syndefald , gruelige Anfegtelfer f . Ex . bespottelige Tanter imod Gud og det Hellige , som det jo hendte Job , David , Petrus m . fl. — nåar nu dcm famme Nod vedvarer i baade Maaneder og Aar . og al den troende Sjels Raab og Klage fur Herren synes at vere frngteslos — da er det et übegribcligi Guds Undervert , at en saadan Sjel endnn tan forblive i Trocn , endnu lan holde sig for et Guds Barn og Gud for sin hulde Fader . Det er iser under saadanne Erfcrnngcr , det bliver nodvendigt at have en hsiere og fastere Grnnd sor Troen end alt , hvad der er i os , synes og foles hos os , en Grund , som er aldeles udenfor og over os . Thi nåar vi paa nevnte Maade bestormes as alle Afgrundens Magter , Synd , Fristelser , Fald . Tvivl , Afmagt — rna sporge vi atter : Hvem paa hele Jorden kan da have en saa urimelig Tro , at ban endnu kan vedblive at holde sig for et Guds Barn , en Kristi Broder , en Himlens Arving ?  Umuligt , urimeligt ! " saaledes maa alt , hvad der er i os , raabe og strigc ; thi hvem kan tro noget saa hoit og herligt , nåar han erfarer noget faa ynteligt og forstrektcligt ? — Her er kun eet Middel , hvorved vi midt i den brusende Flod as alle bedrovclige Erfaringer endda kunne forblive i Troen , nemlig det , at vi bestandig og uafladelig ved Ordet feste vort <b>Oie</b> paa , hvad Gud felv har gjort og fagt om vor Slegt , hvad Gud <b>selv</b> i Evighed har besluttet samt i Tiden fuldbyrdct og fortyndt . Det bli / ver nodvendigt bestandig at ihukomme Grundene , hvorpaa dette Barnestab stotter sig . Dersom jeg f . Ex . paa en Reise seer tet ved Veien en stor , stjsn Eiendom med et pragtfuldt Pallads , omgivet asal Naturens og Knnstens Rigdom og Glands , og jeg strax derpaa <b>moder</b> en fattig Vandringsmand , hvis urene , forstidtc Kleder og blege , afmagrede Ansigt vidne om allestags Elendighet » , og en Sen af mig , forn var hjemme paa Egnen , vilde sige til mig : „ See paa denne fattige Vandringsmand ; troer Du , at denne Usling er Eier og Herre til den store , stjonne Eiendom , vi saa ocrhennc ? " saa maa jeg uviltaarlig udraabe : „ Det er umuligt , det er Skjemt ! " Men min Ven svarer mig leende : . . Jeg forundrer mig itkc et Oieblik over , at det fynes Dig umuligt , saalenge jeg ikke har meddelt Dig de selsomme Forhold , hvorpaa det grunder sig . Denne usle Vandringsmand er nemlig just nylig hjemkommen fra et fremmed Land , hvor han i lang Tid har tumlet om forn en fortabt Son til fin Faders Sorg . Han er der dels bleven plyndret og ilde slagen af en Rover og har dels <b>sclv</b> odelagt sig i alle Henseender ved et lastcfuldt Levnet , saa at han derover seer saa ilde ud , som Du nu saa ham . Men han er i <b>Sandhed</b> encste Son og Arving af denne store , stjonne Eiendoms Grund legger , og denne hans hulde Fader har tor sin Dod med blodendc <b>Hjerte</b> oprettet ct Testament for fin faldne Son , hvorved

<h4>243</h4>Born — itke blot til en vis Kristen , som har gjort Dig noget Godt eller forstacer at vurdere Dig eller har nogen naturlig Elstverdighed, som indtager Dig — men idethele alle dem , som Du seer at elske eller soge lesum . saa at disse nu ere Din nye Slegt , Dine Sodstende — vid , at blot dette Forhold beviser meget mere end alle de herligste Gjerninger , at en ny Fodsel es Guds Aand er gaaet for sig i Din Sjel . Du kan maastee ellers ikke synes , at Du tan vere noget Guds Bant , efterdi hele Din Kristendom fynes Dig at vere altfor mangelfull ) ; men alle disse Mangler og dette Dtt Tykke betyder intet imod Kristi Ord om dette Tegn . — Derimod . om Din Tilstand er saaledes , at Dn vel er bleven vakt af Din gamle Syndefovn , og Du har omvendt Dig fra Dine vilde Stier til Gudfrygtighedens Vei , til Guds Ord , til Bon og Bod , men Du har den Egenskab , at Du ikke vil have noget at gjore med andre naadefogende Sjele , men Du trives bedst alene , som Du siger , „ med Gud og Order , " saa synes dette vist at vere meget fysnt og acmdeligt , men det har blot Et imod sig . nemlig at det strider aldeles imod Hovedtegnet paa den rette Guus Naade . og Din Omvendelse er visselig falst og selvgjort . Dersom Din Omvendelse var et <b>sandt</b> Aandens Verk , da havde den ogsaa dette Tegn , <b>Kjerligheden</b> ttl Brodrene , da stulde Du vere saa nedslagen over Dig <b>sclv</b> , at Du i Almindelighed ansaa andre naaoessgeude Sjele for bedre eno Dig felv , og om Guds Lams Blod var vlevct Din eneste Trost og Bcrsmmelfe , om Du havde veret i den „ trange Port , " hvor baade Din Synd og Din Retferdighed vare blevne tttinretgjorte ved Guds Sons Blod , og Dit <b>Hjerte</b> var blevet ret faliggjort ved den overflsdige Naade — kortelig , om Du levede i den Omvendelfe, hvor Du felv aftog , og Kristus tilvoxede , da skulde Du i denne store oa for alle felles Gave sammensmelte saaledes med alle Medarvingene , at Du i <b>Sandhed</b> blev „ eet <b>Hjerte</b> og een Sjel " med dem , ligesom de forste Kristne vare . — Me videre , om Du end har nogle visse Brsdre , som Du synes om , fordi de ere enige med Dig i noget vist , for Dig meget magtpaaliggende Sporgsmaal , meu det er ikke Fsdfclen af Giw , det er ikke den store Naade i Kristo . som forener Eder , men nogen anden Overensstemmelse , da er dette ikke den rette udmerkende <b>Kjerlighed</b> ; thi lohcmues siger udrrykkelig : Den , som elsker ham , forn fodte , elsler og den , som er fodt af ham ( 1 Joh . 5 , 1. ) hvormed han lader forftaa , at den rette <b>Kjerlighed</b> til Brødrene har sit <b>Blik</b> derpaa , at de e ^ e fodte af Gud , og elster dem just derfor — som han ogsaa strax tillegger : Derpaa tjende vi , at tn elsle Guds Born , nåar vi elfle Gud og holde hans Bud ( v. 2 ) det er , nåar vi elste dem for den felles kjere Faders Skyld . Den , forn ikke giver Agt vaa denne Omstendighed , synes virkelig at ville bedrage sig <b>selv</b> og skal ogsaa blive bedragen. — Hvor herligt derimod , nåar Du veed med Dig <b>selv</b> , at oet just er for Naadens Verk i Hjerterne , at Du elster Brsdrcne

<h4>392</h4>han har indtryktet i Dit <b>Hjerte</b> : Det er nol , evigt nol , hvad Jesus har lidt og gjort . Og den , som i sin Elendighet ) har faaet sit Alt og nok i Kristo , den er frelst , den er salig , den har Troen , oen er en Kristen , den kjender Kristus og har evindeligt Liv . „ la , " siger mangen En , „ jeg veed alt dette om Kristus , holder det ogsaa for <b>sandt</b> ; men jeg faaer ingen Kraft deraf i Hjertet , det vtrker ikke det Liv , den Trost , Fred , Glede og Kraft , som det stnlde " osv. Saaledes klage Mange . Og det er <b>sandt</b> , „ Troen er ille Alles " ( 2 Ths . 3 , 2 ) . Mange have denne Viden om Kristus , denne historiste Tro , men have aldrig smctgt den Listighet » og Kraft , forn den sande , levende , tilegnende Tro har med sig . Andre derimod udtrykke med denne Klage blot en Trost efter Fslelfer og liflige Fornemmelser af Naaden , af Frelserens <b>Kjerlighed</b> og Nerverelse . De have virkelig Tro , dave nemlig i Kristo deres eneste Trost og Skat , deres Alt i Alle , have ogsaa ved Troen sacret et nyt Sind og vide tydeligt at skjelne imellem den Tid , de stode under Loven , og denne under Naaden ; men de ntindes ogsaa , hvilke listige Folelser dc i Begyndelsen havde , og torste nu atter ester saadanne . Disse burde engang betenke , at hele Skriften taler blot om Tro , Tro paa lesum Kristum . og aldrig om Folelser , om at tro paa selve Ordet , som det lvder , og holde sig dertil , Folelsen maa vere listig eller bitter ; oe burde betenke , at de bor oog vere den gode Frelser til Behag deri at lade sig nsie , om det behager ham at fordslge sig og prsve deres Tro , som han gjorde med den kcmctneiste <b>Kvinde</b> , ' og ihukomme , hvor kjert det var for ham , at hun endda troede , faa at han just havde en hjertelig Lyst deri og ndbrsd: D . <b>Kvinde</b> ! Din Tro er stor , Dig slee , som Du vil ! og da fik hun ogsaa at smage og see , hvor listig t ) an var . Lad os gjsrc Frelseren denne Glede ; han har ingen stsrre Glede af os , end nåar vi tro paa ham ! Men virkelig at tro , det er , i al Bekymring over sig <b>selv</b> , i al Usselhed , Synd og Anfeqtelse dog virkelig at have i Oristo sin Trsit , sin Klippeborg , sin Retferdighed og Fred , det er aldeles nsdvendigt . Og for at vi maatte faa demte Kraft af Ordet , for at Gud , som har Troens Gave i sin Haand , maatte kunne give os en virkelig og levende Tro , saa er det i Serdclcshed nsdvendigt , at vi stille og enfoldigt give Agt paa noget vift Stylle af Guds Ord . noget vift Kjernefprog om Kriftus , stille dybt og ctlvorligt betragte det ; thi „ Troen lommer af Prediken." Mange forspilde Guds Naade ved blot bestandig at vende Tankerne hit » og did , om end paa aandelige Ting , og kunne ikke en eneste Stund holde Oinene stille paa den Korsfestede. De ville ikke tro , hvad Skriften allesteder vidner , at Alt , Alt kan hjelpes ved Troen , at Kristus er os given baade til Retferdighed og Helliggjorelfe , derfor have de saa mangeslags Omsorg , at Ordet aldrig naaer til at synke ned t deres Hjerter

<h4>612</h4>om han blot kunde bestrive , hvad der er ham <b>selv</b> saa kjert , da stulde ogsaa Andre sorstact det , see hvad han seer , og tomme til <b>Sandheden</b> ; da taler han itke med kunstige Ord efter Menneskers Visdom , men med den elskelige Enfoldigheds Sprog tan han nu paa naturligste Maade og af egen Erfctrmg beskrive f . Ex . Syndefsvnen, Siklerhedens Stotter og vrange Meninger , Guds Ord og Domme , Synden og dens Folger eller den store , uforskyldte Naade i Kristo , samt hvor stor og inderlig Glede det er at vere bekjendt med Frelseren osv. sandes Luthers Ord , at . . Evangelium soder ittc saadanne Born , som ligge stille i Vuggen , men raste Anglinge , som ogsaa ere dygtigc tit at omvende Andre ; " thi „ de stulle alle vere fodte af Gud baade Smaa og Store , " og af denne Aandens Lerdoru taler da en ellers Ulerd langt bedre om de guddommelige Ting end dc mest Hotlerde , som endnu <b>selv</b> ere uomuendte og uerfarue . Meu Mctctden , Gaverne vg Kaldclserne ere forskjellige ; thi ikke alle Lemmer i Legemet have samme Embcde ; <b>Oiet</b> seer , men kan ikke hore ; Fodderne gaa , men kunde itke tale ; Tungen taler , men gaacr ikke osv. Den Eve har den Gave at tale med egne Ord til Sjetes Opdyggelse af Gnds Ord , venlig og viselig at rctade , advare , vekke , op ! yse og beskrive de aandelige Ting ; den Anden har „ et tungt Mele " ligesom Moses , men lan vere virksom til at lese i Bog , paapege vigtige og vekkende Stykker ; en Tredie taler meget tydeligt med sin alvorlige , gudelige Vandel, hdormed ogsaa alle Troende med mere eller mindre Held umage sig for at predike , hvurvcd de beftraffe og — forn ogsaa Verden erkjender — virkelig . . domme , " ogsaa da , nåar dc ei sige et eneste Ord med Munden ; den Fjerde er et uophorligt bedende <b>Hjerte</b> i Kristi aandelige Legeme , som paa denne tause Maade med Forbsnner gjor Guds Gjerning ; den Femte er motter og virksom ved gode kristelige Indretninger og Stiftelfer . Dette fynes i vor Tid at svare til Apostelens Ord : „ En gives Visdomstctle formedelst Aanden , en Anden Kundstabstale ved samme Aano — en Anden Tro ved den samme Aand , en Anden Gaver til at heldbrede — en Anden Kraft til Undergjerninger , en Anden profetisk Gave " osv. Herved maa dog merkes , at dette bor ikte forstaaes saaledes . som om Enhver kun kunde virke paa een Maade og ingen Del have i den Andens Gave ; men her omtales kun at Enhver har sin ndmerkende Gave ; f . Ex . den , som savner Gave til i Almindelighed at lere og undervise med Ord . og har en treg Tunge , han kan dog tale noget , kan dog ligesom de bethlehemitiste Hyrder . . kundgjsre " paa Fctctrehyrders simple Sprog , hvad de have erfaret om Barnet , eller forn <b>Kvinderne</b> forkynde , hvad de have feet ved Jesu Grav , lobe med en Hilsen fra den opstandne Frelser og sige forgende Sjele en glad Tidende , ihvorvel det steer paa et <b>kvindeligt</b> og enfoldigt Sprog . Saa maa de jo og alle predike med Levnet , og Alle maa jo

<h4>623</h4>sin endnu hedenske Son , saa svarede Bistoppen : „ Det er nmuligt, at en saamange Tcmrers Ssn kan fortabes " . Og hans Ord bleve Scmdhco paa Augustin , som blev en merkverdig Kristen . Vi have nn i nogen Maade seet . hvad vi som aandelige Prester have at gjore , og saa staaer nu blot tilbage , at sette det i Verk . Det er <b>sandt</b> , at en Kristen , som daglig toer sine Kleder i Lammets Blod , daglig hos Gud beklager sine Forssmmclscr og Synder og soger Nactdc ved Jesu Forsoning , han har visselig en bestandig Forladelse for alle Mangler og Forssmmelser ogsaa i dette Tilfelde , ogfaa for disfe forstrekkcltgc Synder at have negtct Menneskenes Sjele den Tjeneste , hatt hctvdc tunnet gjore dem med Ord , Gjerning og Forbon , ja at han endog med sine Synder har givet Forargelse og hindret Gnds Verk — alt , alt er det i Kristi dyre , evigt gjeldende Blod forsonet ; men dcn , som har imodtagct og trocr sacmtcgen Naade , han drage den itke ' til Losagtighco ! han vogtc sig for ogsaa fremdeles at forfomme dc hellige ' Pligter , som just denne store Naade desto kraftigere fordrer ! I „ den forftc Kjerlighcds " Iver og Varme indblcmder sig visselig ofte meget Uvifeligt . forn bor afleggcs ; men maatte blot ikke Visdommen blive en saadan Klogstau og Forsigtighed . som fortrenger den forftc <b>Kjerlighed</b> ! Mange falde undertiden i en Treghet » og Forsommclse . som er langt verre end den forftc Kjerlighcds uvisi Iver . Det er fra denne Forfommelse , Kulde og Sovnagtighcd , vi atter bor opvekkc hverandre med Guds Ord , ja med dc rene , enfoldige Ocvegarrmdc, forn i de forftc Tider oplivcdc os faa virksomt . Om Du f . Ex . bar fovet . men bliver vekket i et cmtendt Hus , som snart brender ned , var det da forfvctrligt , om Du kun vilde redde Dig felv og ikke forfogtc at vekke dem , forn endnu fov i famme Verelfc ? Ont Du nys har ligget nede i en fvag og bristet Is , mcn endelig cr bleven frclft , stuldc Du da kunde fee Din Nabo strax derpaa at gaa ned paa famme Is nden at advare ham og sige , hvad Du <b>selv</b> har erfaret af dcv ? Nu er Du maastee Fader , <b>Moder</b> , Husbonde , Du har Born og Tjenestefolk , Du har Naboer og Venner , med hvilke Dn daglig omgactcs ; Dn er fclv ved vor Guds uendelige Barmhjcrtighcd ndrcvcn af det Morke , den Sikkcrhcd og Syndcsovn , hvori vi ctllc hctvc ligget; Du seer disse Dine Omgivende at ligge der , hvor Du just <b>selv</b> har ligget — stuldc Du uu ikke ville gjorc uoget til deres Opvekkclsc ? Skulde Du ikte forsogc med et vcnligt , ydmygt og kjerlighcdsfuldt Samrctad at fore oem til Besindclsc , eller ved at laane dem en god Bog give dem et Vekkclsesmiddcl ? Eller har Du <b>Hjerte</b> til at lese og betragte Luk . 7 , 34 — 36. hvorledes Kristus forudsigcr , at den . Nat . da han kommer over os , stulle  To ligge i een Seng " , . saasom Mctgcr eller Sodstcnde , den enc stal ' annammes , og den anden lades tilbage " — „ To stulle male paa een . Kvern , den enc cmnammcs og den

<h4>753</h4>med vort Levnet t Kristi Sted mcd Ord og Gierningcr , enten vi tro og betcrnke det eller ikke ; thi det gaar dog altid , som iKristus siger , at han bliver irret , om vi berre megen Frugt illoh . 15. ) men derimod , at hans Navn bliver bespottet , om vi ' leve stjodeslost og ilde ( Rom . 2 , 4 ) . O , hvittet betcrnkeligt Forhold, o hvilken forstrcekkelig Synd og Skade , om vi forglemme dette og leve , fom om vi vare vore Egne og kunde gjore , hvad vi ville ; — Kristne ! vil Apostelen sige , bctamker , at I ikke . cre Eders Egne ; . l ere Kristi Lemmer , Eders <b>Hjerte</b> er hans Tempel, Eders Tunge er Kristi Tunge , Eders <b>Oine</b> , Eders Hcrndcr , alt er Kristi Lemmer ! Alt . hvad Inu gjore med Ord eller Gjerning , det gjorer alt i den Herres lefu Navn . i hans Sted , ltgesom han <b>selv</b> var her og gjorde det ; gjorcr Alt som hans Lemmer til hans VEre og Velbehag paa den Macrdc , som det vilde cmstact ham <b>selv</b> at gjore , hvad I gjore . Eller hvent stulde vove at gjore Synd mcd Kristi Lemmer , at tale unyttiac Ord med Kristi Tunge , gjore onde Gjerninger med Kristi Legeme ! Derfor gjor Intet , forn det ci var passende for Kristus at gjore , taler og gisrer saaledes , som han tctlede og gjorde ! Han var mild og ydmyg af Hjertet ! det var hans Mad at foge Faderens TEre og Velbehag , Naade og <b>Sandhed</b> vare altid paa hans Lcebcr ; vcerer hans Efterfolgere heri . Kortelig , Alt , hvad I gjore med Ord eller Handling , det gjorer alt i den Herres lefn Navn . „ Om Nogen taler , han tale forn de Guds Ord , om Nogen har et Entbeoe , han tjene efter den Evne , Gud giver." Den , forn fclvist foretager Noget og gjor det til sin egen Mre , efter sit eget Tykke og Velbehag og i egen Kraft , han gjor det i fit eget Navn , og da er Alting tabt , han har faaet sin Lon ! — Nei , gjorer Alting i den Herres Jesu Navn ! Takkende Gud og Faderen formedelst ham ; Giver Gnd Wren for alt Godt . Kjender og erkjender , at I ikke engang formåa at tcenke Noget forn af Eder felv , men ere vi dygtige til Noget , saa er det en Gave ved Kristus . See dette er nn Summen af alt kristeligt Levnet . Men alt dette bliver knn Ord oa Tanker , bliver aldrig Gjerning og Virkelighed , om ikke Kristi Ord rigelig og med al Visdom kommer til at bo i os . Derfor bliver Slutningen denne : Der er intet Middel til altid at forblive i Troen og <b>Kjcerligheden</b> , i en falig og fortrolig Forening med Frclferen famt i Kicerlighedens Lyst og Kraft til et kristeligt Levnet uden dette eneste , daglig at komme tilbage til Ordet , lcrse , tale stride og synge om Kristus , lcere og formane os felv og hverandre alle Dage ; derved stulde alt Ondt ctfhjcelpes , og alt Godt gives os . Mod Synder , mod Vcmtroesrvaler , mod Lctsindighed , mod ctllchaande Vildfctrelser og falske Aano er , kortelig , mod alt Ondt hjcelper kun et Eneste : Krlfti Ords flittige Brug . Der findes ingen faadan evangelist Visdom , ingen saadan Troens Methode , at vi deraf

<h4>1050</h4>Hvad siger Skriften herom ? Skal der endda vere Spsrgsmaal om Din Retferdighed eller om de iboende Synder , stjsndt Du har flyet til Kristus , saa at Du er knyttet til ham for hele Dit Liv ? Skal da Kriftus ingen Tjeneste gtsre dem , som tro paa ham , stnlle de ikke ved ham vere sri for al Tilregnelfe af Synd ? ( Nom . 4. 6. 8 ) . Men er det itte <b>sandt</b> , at med alt Dit Evangelium og Din Tro er der dog En . som i Stilhed tilhvisker Dig , at Du stal ogsaa <b>selv</b> vere frr fra hver virkelig Synd , og at Du ellers er falst , er en Hykler , en Slave m . m . og Kristus er Dig itte nyttig ? Ja det gaacr faalangt med den Indgivelfe , at vi rentud forvildes i Forstanden . Her bliver det da hoist nodvendigt , at holde <b>Oiet</b> haardt festet ved , hvad Gud felv siger , og vogte sig som for felve Djevelen for at fpsrge efter Folclfen under Provelsestider . Har Du itte af alle de Helliges Exempler feet , at det just tilhorer Guds Maade paa at ove Troen , at han stiller sig saaledes imod dem , som om han aldeles intet vilde dem , som om de vare de storste Syndere paa Jorden , og som om han itte var saa vred paa noget Menneske under Solen som paa dem ? Hvorledes var vor Vei en Troesvei , om han itte gjorde saaledes ? Vogt Dig da for Folelfen og de omtringflyvcndc Tanker ! Gjor enfoloigt blot et eneste Sporgsmaal : Kan jeg faa Din Naade , o Gud ! eller vil Du vredes evindelig ? Gud . faaer ieg Tin Naade ? Og hvad svarer da Gud ? Vogt Dig for at vente Svaret i Dit <b>Hjerte</b> ! thi der faaer Du Provelfens Svar , forn den lancmeifle <b>Kvinde</b> sit ( Math . 15 , 23 — 26. ) men Du stal tåge Svaret alene af Stnftens Vidnesbyrd om Gud ! Hvad fvarcr da Gud , nåar Du blot fporgcr : Faaer jeg Din Naade .

<h4>1132</h4>terknnse Slangens Hoved , bere Verdens Synder ( Es . 53 , ) og i hvem alle Jordens Slegter stnlde velsignes ? Her , imod disse Aartustndernes li tallige Vidnesbyrd , blive dog mit <b>Hjertes</b> morke Tvivl og Modsigclser som dc for Vinden flyvende Stov og Stråa imod det gamle faste Bjcrg . Gud Herren vere evig Lov og Tak og Pris for sin uudsigelige Gave . Da „ sendte Gud sin Son . " Her er nu sclve Hjertcpunktct i vort Sprog og i hele vor kristelige Tro . Men her er ogsaa det Punkt , som vi aldrig ret kunne faa ind i vort <b>Hjerte</b> . Her er det Punkt , hvorover man blot bnrde anstille Bonner med megen Paakllldelfc , at Gud vilde forbarme sig over os og aabne vort Sind , at vi maatte kunne sec og stnc Noget i de umaadclige Dyb af Guds Naadeunder , da han gav vor arme faldne Slegt sin egen Son . Lad os see , hvorlcdcs Ortene lyte . Det hcddcr : Da sendte Gud sin Son . Merk , der stacer ikke : Da fodtes Guds Son , men : Gud „ fendte " fill Son , fodt af en <b>Kvinde</b> . Det Barn , forn Du i Aanden stuer i Krybben , er ikke et Barn , forn har faaet et saa stort Maal af Guds Aano og Guds Velbehag , at Gud derfor talder det sin Son — nei , anderledes taler Skriften: den siger nemlig saaledes , at det Barn var Guds evige Son . som „ var hos Faderen , ssrend denne Verden blev stabt , " den siger , at i Bethlehem stulde fodes den , „ hvis Udgang er fra fordum , fra Evigheds Dage " ( Mik . 5 , ) at  i Begyndelsen var Ordet , og Ordet var Gud , saa at ved det er Alting stabt , og uten det er intet blevet til af alt det , som er , < ^ og Ordet blev Kjod og bocde iblandt os , og vi saa hans Herlighed som den Enbaarnes af Faderen , fuld af Naade og <b>Sandhed</b> . " Saadant ligger i det „ fendte " — Gud feudtc sin Son , altsaa en Son , som forud var til , men forn nn i Tidens Fylde alene blev fendt til Jorden . Det siger og Kristus <b>selv</b> : „ Saa elskede Gud Verden , at han hengav sin enbaarne Son . " „ Fc > ocr , forklar Din Son mcd den Hcrlighcd , foin jeg havte hos Dig , forend Verden var . " „ leg udgik fra Faderen og kom til Verden; jeg forlader nu Verden og gaacr til Faderen . " Saa siger og Apostelen Johannes : „ Dcri er Glids <b>Kjerlighed</b> til os aabenbaret , at Gud har sendt sin enbaarne Son til Verden , at vi skulle leve ved ham — hali clstcdc os og udsendte sill Son til en Forsoning for vore Synder . " Og ' atter Paulus : „ Det gjorde Gud , da han sendte sin Son i syndigt Kjods Lignelse." See , Saadant stal Dn forståa , nåar ' Du i Nauden sidder vcd Krybben i Bethlehem og ssner det merkverdige Barn , hvis Fodsel bcbndes af hele den himmclste Herskare for fattige Ayrdcr paa Marken og af den undcrfuldc Stjerne for vise Ment , i Osterland . Tak og Lov og VEre og Pris og Styrke vere vor Gud fra Evighed til Evighed for hans undsigclige Gave ! Gud fendte fin Son . Fodt af en <b>Kvinde</b> . Übegribelige Himmclunder ! Guds

<h4>1134</h4>evige Son bliver Menneske , et virkeligt Menneste , kun ikke et syndigt , men et rent Menneske . „ Foot af en <b>Kvinde</b> . " Det , som er ssdt af en <b>sand</b> , naturlig <b>Kvinde</b> , maa vere et <b>sandt</b> , virkeligt Menneste ; en virkelig Kvinte soder intet andet end et virkeligt Menneske . Saa er da Guds evige Son virkelig et Medlem a f vor Sllegt , saa at hans Stamme og Slegtninge regnes iblandt Adams Eftertommcre , som ogsaa det forste Blad af det nye Testamente viser . Guds Son er vor Slegtning , vor Vlodsforvandt, „ Brodrene lig " undtagen i Synden . Übegribelige Guds <b>Kjerlighed</b> , fom ^ fattede et saa besynderligt og naadefulot Raad til vor Frelse . Men her morkner det dog for Fornuftens <b>Oie</b> . Guds evige Son , vor Vlodsforvandt , vor Broder ! Er det fandt , er det muligt? Jeg begynder , og jeg slutter , og jeg begynder igjen ved dette Punkt . Er det <b>sandt</b> , er det muligt ? Jeg lav ikke begribe det , men jeg kan heller ikke komme fra det . Mine Tanker befrieres som i et Net af evige Übegribeligheder , men ogsaa uundgacelige Vidnesbyrd . Og Sporgsmaalct har den Vegt , at med det staaer eller falder hele Kristendommen og med den dor Frelse og Salighed . Paa den ene Side er det altfor stort og herligt , at vi arme faldne Menneskeborn stulle vere faa hoit skattede og hedrede , at Guds evige Son stal vere vor Broder og Vlodsforvandt — det er altfor stort og herliat , saa at jeg itte kan tro det for dets Storheds Skyld ; paa den anden Side er denne Anstodssten mig altfor fver at bortkaste . Der stacer en hel Verden fuld af Vidnesbyrd om ham , forst en hel lang Verdensalter mcd besynderlige Forudsigelser og siden en efterforgende Verdensalter fuld af besynderlige Opfyldelser , dc mest unoerfnlde, punktlige Opfyldelser af alt ' , hvad der var forudsagt . Kan jeg helt forkaste den Person , som er Fonnaalet for alle disfe Forudsigclfer og Oftfyldclfer ? Den Sten , som Bygningsmendene forkastede , men som Gud dog gjorde til en Hovedhjornesten, stal ogfaa jeg forfoge at forkaste den ? Eller kan jeg forkaste , blot , hvad jeg feer i denne Dag af hans Rige paa Jorden , ja tan jeg forkaste , hvad jeg <b>selv</b> har erfaret af den trofaste Herre ? O nei , han er en levende og nerverende Gud , vi kjende ham jo . og dette blot som aabcnbaret i Kjod ; thi Ingen kjender Faderen uden Sonnen . Men da — kan jeg da paa den anden Side tro alt dette Store og Herlige , som tigger deri , at Gnd er aabenbaret i Kjod ? Det er jo dog altfor ftort og herligt ! Og dog er det jo en evig , guddommelig <b>Sandhed</b> , en Sandhed, som er lige vis og guddommelig , om end alle Menneskers Fornuft gaaer tilgrunde derpaa . Ja Gud vere Lov , at vi ikke tunne begribe den , dct er , at vi ikke have en saa liden Gud , at vor arme blinde Fornuft kan fslge ham . Gud bevare os fra at have en Gnd , som ikke stulde overgaa vor Forstaud , der jo ikke engang kan fatte vort eget Vefen . Men at det arme faldne og trange <b>Hjerte</b> ikke kan beholde

<h4>1304</h4>bliver Gjerning og Sag og ikke blot fromme Tanker , Forestillinger, Indbildiiinger , Ord og Forsetter . Naar f . Ex . et Menneske, som endnu er udeu Troens Liv , uden den Helligaand , soger Gudfrygtighed og Helliggjorelse , saa tenker og taler han adstilligt smutt og nidkjert om Forsagelse og Ydmyghed m . m . umager sig ogsaa med at forsage , og forlader mange Ting , som han for har elsket , og som tilhore Verden ; men fee , til famme Tid har han en hemmelig Tilfredsstillelfe , ja sit Haab og sin Trost i denne sin Forfagelfe kan vere fuld af Selvkjenighed og Indbildning om sin egen Fromhed . Hvorledes er det da med Forfllgelsen ? Der sattes selve Hovedsagen , sattes det fsrste og fornemste Stykke as en ret Forsagelse , som cr at forfage fig felv ; thi dertil horer at fynes virkelig ilde 0 M fig felv , virkelig mistruste paa sig <b>selv</b> . Han kan vel misbillige visse af sine Vanartet, og visse daarlige Gjerninger , men sig <b>selv</b> elster han , saalenge han kan sinde Tilfredsstillelse i noget af sin egen Bodssvelse. Mange ville ikke forståa fig paa Forskjel ' mellem disse luende Ting , at vere misfornsiet med vissc ch sine daarlige Gjerninger eller Vanartcr og dcrimod at vere misfornoiet med fig felv , holde fig <b>selv</b> hel ug holden for en fordsmmelfesverdig , foragtclig Synder ; men det er dog just dette , forn egentlig stiller den sande og salste Omvendelse fra hverandre . . At saaledes rettelig forsage sig <b>selv</b> , at synes virkelig ilde om sig <b>selv</b> , det kan Ingen nden den Helligaand virke , det steer alene i den sande Omvendelse , da Gud forst ouerhobcr Mennestet med faadanne Lovens Krav , forn han ikke kan opfylde , mcn dog alvorlig umager sig for , og derfor omsider mistroster paa stq <b>selv</b> og alle sine Foretagender , og dernest overhoder ham med Aacide ved Evangelium,' uaar han faaer Troens <b>Oine</b> aabncde og fecr , hvorledes han har faaet dobbelt af Herrens Haand for alle fine Synder , feer , hvorlcdcs Gnd midt i den storste Uverdighcd forlader ham Alting — indre og ydre Synder , forbigcmgne og nerverende Synder , grovere og finere Synder — rentvetter og rctferdiggjor ham uden ringeste Fortjeneste alene for Kristi store Forsonings Skyld ; da smelter han , da nedsynker han i inderlig Blusel over Lg <b>selv</b> , da kan han ikke nok foragte sig <b>selv</b> , da forst begynder den rette Forsagelse , da er der <b>Sandhed</b> i hans Forsagelse . Dette gjsr <b>Sandhedens</b> Aand . Videre taler en uden Tro og Aand arbeidende Sjel om <b>Kjerligheden</b> : Man maa have <b>Kjerlighed</b> , man bsr elste Gud og sin Neste ; hvem stulde ilke elste Gud ? osv. Mcn see , om et saadant Menneste kommer til et lidet Oiebliks Bcsindelse og faaer Naade til ret at opvaagne , saa maa han beljende , at han endnu aldrig har faaet denne <b>Kjerlighed</b> i sit <b>Hjerte</b> ellcr i Virkelighed , om han cnd i syrrctyve Aar har arbeidet derpaa , men at det endnn er dsdt og koldt i hans <b>Hjerte</b> , dcr er ingen <b>Kjerlighed</b> , Liv , Varme og Salighed . Al hans Tale om <b>Kjerlighed</b> har saalcdcs blot veret Ord i Munden , blot Tanker og Forsetter , ingen

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112101047'target=_>, 1857, De fire Evangelier og Apostlernes Gjerninger</a><h4>558</h4>saa var dette dog heller Me for dem noget aldeles Nyt og Übekjendt ; ogsaa disse hans Salighedspriisninger pege tilbage paa Udsagn i det G . T . ( se Es . 57,15 . 61,1 - 3. Ps . 34,11 . 19. 37,11 . 73,1 . 1 Sam . 2.5 . Ps . 51,19 . Prcrd . 7,4 . v. a . ) , og paaminde derfor hans Tilhsrere om bekjendte <b>Sandheder</b> . —  Han begynder med sande Gjerninger og stiller dem imod Pharisceernes falske . Ligesom vilde han sige : Gode Gjerninger lcrres af Phariscrerne , men blot for Syns Skyld ; man maa gjorc gode Gjerninger i Aand og <b>Sandhed</b> , det er , af et yd « mygt , sagtmodigt , taalmodigt , hungrende og torstcnde , fredsommeligt , recnt , forbarmende <b>Hjerte</b> . " Luther . — Derpaa fremstiller han i Korthet » Hovedindholdet af Talen : den dybeste , fuldstaendigste Opfylvelse af Loven er Endemaalet for Christi Aabenbarclse ; kun den , der straber derefter , kan have Dccl i Himmeriges Rige ( V . 17 — 20. ) . - Idet fsrst Johannes og senere Jesus <b>selv</b> overalt optraadtc med Fortyndelsen af , at Guds Rige var kommet , alle Provhttier opfylvte , saa maatte i Begyndelsen hos mange Kjodeligsindede det Haab vaagne , at nu var Loven ophcevet . Til disse Onsker og Forhaabninger stiller han sig i skarp Movsa-tning , og viser , at Loven ved Evangeliet ikke ophaves , men oprcttes ( Rom . 3,31 . ) . Paa det at Loven nu maa kunne opfyldes i sin hele Dybde , afklceder han den alle de Skranker , i hvilke deels Guds Huusholdning med Menneskene i den gamle Pagt , paa Grund af Hjerternes Haardhev og Umyndighed , deels ogsaa Pharisaeernes udvortes Voastavtjeneste , havde indestuttet den ; og idet han aabenbarer Gud som alle deres Fader , hvilke han optager i sit Rige , er den taknemmelige Tilegnelse af hans Fullkommenhet» Maalct for hans Born ( V . 20 — 48. ) . — I Modsa-tning til Phariscrernes , viser deres Sindelag , der ere Himmelrigets Medlemmer, sig deri , at ogsaa i alle deres Handlinger , navnlig i de , som dengang gjaldt for Indbegrebet af al Fromhed , Guds Velbehag og Samfund er deres eneste Formaal ; derfor ere deres Almisser Åttringer af stille , forborgen <b>Kjcerlighed</b> , deres Bsn Hjertets Sag . de ydre Tugtemidler ( Faste ) blot bestemte til at virke indvortes ( C . 6,1 - 18. ) . - Dertil knytter sig en , hvad det hovedsagelige Indhold angaaer , ogsaa paa et andet Sted holden Tale angaaende det at tragte efter himmelske Goder , som det eneste Maal for al Strcrben og Bekymring ( 6,19 — 34. ) . Jesus gaaer ud fra at vise den sande Skat i Himmelen , hvilken man ikke kan see uden et enfoldigt sundt <b>Die</b> ; og viser derncest , at det ikte er muligt , men heller ikke nodvendigt at tjene to Herrer , eftersom vor Fader i Himmelen sorger saa rigelig for os . Derpaa folge Formaninger til Himmelrigets

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014090108027'target=_>Rosenius, C.O., 1868, Hemmeligheder i Lov og Evangelium</a><h4>267</h4>en Borttastelse af det eneste os givne Frelsens Middel , af det eneste Offer for vore Synder . Gud har i sin inderlige Barmhjertighed forbarmet sig over vor Elendighed og givet os sin enbcmrne elskelige Son til Frelser ; han kom og blev vor Broder , blev et Menneskebarn og hengav alt , hvad han havde , sin Retfcerdighed , sit Liv og Blod til vor Frelse ; Dn veed alt dette og er dog altid lige kold og ligegyldig for ham ; Du feirer hans Fodsel om Inlen , lceser , horer og synger om Barnet i Krybben, Du feirer hans Lidelses- og Dodsfeit , lceser og horer om hans Blodsved i Urtegaarden , hans Hndstrygelse , hans Tornekroning, hans Hcenders og Fodders Gjennemborelse , Du seer ham ophcengt i Luften mellem Rovere , Du horer hans trosteslose Angestrcmb , og Du veed , bekicnder og synger , at alt dette skete „ for Dig og Dine Synders Styld ' " Du gaaer til Nadveren og feirer hans Dods Ihukommelse og med alt dette forbliver Du lige dod og kold i Dit <b>Hjerte</b> , elsker og omfavner ham ikke , gloedes og priser ikke , men gaaer med <b>Hjerte</b> og Tanker fulde af Forfængelighed , Synd og Ulydighed mod denne naadige Frelser . Undres Du da over , at Du maa voere fordomt ? Undres Du da over , at Guds Vrede brcender som fra det nederste Helvede over en saa forskrækkelig Utaknemmelighed mod hans elskelige Sons brcendende <b>Kjcerlighed</b> og bittre Marter , en saa kold Foragt og Nedtrampelse af Guds allerstorste <b>Kjcerlighed</b> ? Luther siger i en Betragtning over Kristi Lidelse : „ Et Menneskehjerte , som ikke bliver bevceget eller rort herved , maa jo voere haardere end Sten , end Jern og Staal . Dog gaaer den kjoere fine Verden sin Vei frem og lcegger dette aldeles ikke paa Hjertet , er trceg , kold , utaknemmelig og foragter denne ftore Skat . Derfor skeer det og , at vor Herre Gud overgiver den derhen , at den kommer bestandig loengere og lcengere bort derfra , og vor Herre Gud gjor just Ret , ncmr han siger til den utaknemmelige Verden : Vil Du ikke see min store <b>Kjcerlighed</b> , at jeg saa faderlig og hjertelig har besogt Dig og hengivet min elskede Son i saa stor Marter , velan , saa vil jeg heller ikke see Dig . Sporger Du ikte efter , hvad jeg har gjort , saa sporger jeg heller ikke efter Dia ; vil Du ikke have min Son , Jesus Kristus , saa tag Barrabas istedetfor , ja Djcevelen <b>selv</b> ! " Ncmr Du er saa kold , haard og utaknemmelig og har aldeles ingen Glcede i Kristus , men viser ham blot Foragt for al hans Naade og Kicerlighed undres da ikke , at Gud lader Dig gaa Din egen Vei tll Fordcervelsen . Apostelen siger : „ Ncmr Nogen har brudt Mose Lov , doer han uden Barmhjertighed; hvormeget vcrrre mene I , at den stal agtes vcerd , som trceder Guds Son med Fodder og ringeagter Pagtens Blod ? " ( Hor . 10 , 28. ) . See her , hvad Vantroen er allerede i sig <b>selv</b> ; derhos er den ogscm <b>Moder</b> til alle andre Synder . Den , som ikke troer paa Krrstus , er bortvendt og afsondret fra Gud , har ingen <b>sand</b> <b>Kjcerlighed</b> , Fortrostning og Lyst til ham og hans Villie , om han end ligesom Trcellen af Frygt eller Indbrldmng

<h4>637</h4>Sce , alt Saaocmt er i sig <b>selv</b> bittert , men stal for Din Guds Velbehag vcere Dig tjcert og behageligt , alene fordr han vil det . Saaocmt ligger der i de Ord : „ elske Gud af Dit ganske Sind " . Thi hvorledes kan man elske Gud af sit ganske Sind og dog ikke elske det , som behager Gud , men fole Uvillie mod et Bud , som han har givet , eller en Lidelse , som han tilsender ? Naar Du kun elsker ' Gud af Dit ganske <b>Hjerte</b> , af Din ganske Sjcel , af Dit ganske Sind , saa skal jo alt , hvad der er ham velbehageligt , vcere Dig kjcert og vcere Dig lige , enten det er lifligt eller bittert , naar blot halt vil det . Og see , vaa denne Maade skulle vi elske Gud . Vi skulle elske Gud saaledes , at vi ikke elske noget Skabt som saadcmt , d . e . for dets egen Skyld , men alene for Skaberens og Giverens Skyld ; men for hans Skyld skulle vi elske Alt , hvad han giver og tilsender os , enten det er lifligt eller bittert , saa at det maatte vcere en <b>Sandhed</b> , at vi elske ham af vort ganske <b>Hjerte</b> , af vort ganske Sind . Og naar vi saaledes i alle hans Gaver elske Giveren og kun elske dem sor hans Skyld , da elske vi egentlig ikte den skabte Ting , men ham , som gav eller tilsendte den . „ Men hvad er dog dette " , turde Nogen svorge , stal jeg ingenting elske nden Guo ? jeg ikke elske s . Ex . min Fader og <b>Moder</b> , min Mage , som med al Omhcd elske mig ? jeg koldt vende mig bort fra mit Barn , som med et elskeligt Smil udstrcekker sine smaa Arme imod mig ? Skal jeg sige til de stjonne Blomster om Sommeren : Blomstrer og dufter frit , jeg maa ikke fryde mig ved Eder ! Skal jeg foragte en trofast Ven , som altid soger mit Bedste , og som saameget bidrager til at lette min Vandring gjennem Jammerdalen ? Kan jeg efter Kristi Lcerdom bede Gud om dagligt Brod og dog vcere ligegyldig for alt , hvad derunder indbefattes , og hvad Herren ester denne Bon giver mig ? " Svar : At opfatte , hvad her er blevet sagt , vaa den Maade , er visselig en grov Misforstand , og med saadcm Misforstand have Mange gjort Kristi Evangelium mistcenkt og bespottet , som om det var egnet til at drcebe al <b>sand</b> af Gud <b>selv</b> given Livsglcede, al Menneskekjcerlighed og deslige . Nei , vi have udtrykkelig sagt , at vi stulle elske Alt . hvad Gud har givet eller tilsendt os , lifligt eller bittert , og dette for Herrens Skyld . Det er saa langt fra , at det forste Bud forbyder den sande Livsglcede eller al <b>Kjcerlighed</b> til Mennesker og skabte Ting , at det tvertimod paala ' gger os at elske Alting , Godt og Ondt , lifligt og bittert , og det blot for Herrens Skyld , blot fordi Herren sender det . Verden elsker kun nogle visse Ting , nemlig de , som i sig <b>selv</b> ere behagelige , saasom Helbred , Velstand , Rigdom , VEre etc. ; men vi bor elske alle Ting , ogsaa de bittre , saasom Sygdom , Modgang , Fattigdom , Foragt og ' Forsmcedelse blot for Herrens Skyld , som sender os Alt . ' Og hvilket Menneske kan have en mere <b>sand</b> og bestandig Livsglcede end den , som elsker alt , hvad der vederfares

<h4>760</h4>herlig Beronunelse at vide og sige : „ Om Du blot r Kjcerltghed og Enfoldighed gjor , hvad det fjerde Bud krcever , saa at Du med villigt Sind og for Herrens Skyld gjor Dine Forceldre eller Dit Fwsbondfolk til Villie i Huset med de ringeste Hvcrdagssysler, saa er dette et behageligere og helligere Levnet for Gud , end om Du Nat og Dag pinte Dig tildode med de svcereste selvvalgte Fxllighedsovelser ! " Thi i ' det fjerde Bud har Du de Gjerninger, som Gud har <b>selv</b> befalet , og vi burde dog vcere forstandige nok til at begribe , at ingen Gjerninger kunne vcere bedre end de , Gno <b>selv</b> har befalet os . „ Hvorledes kan Du vere saligere eller leve helligere , hvad Gjerningerne angaa ? og desuden har Du den Forsiktrmg af Gud , at det stal tjene Dig til alt Godt , og at det skal gaa Dig vel . Saaledes har Du alt Godt , Beskyttelse og Bestjcermelse under Din Herre , en glad Samvittighed og en naadig Gud , som stal gjengjelde Dig hundredefold ja Dn er endog herved en fornem Person , efterdi Du gcmer i Guds Mrinde og tjener Gud <b>selv</b> . " ( Luther ) Men desvcerre er Fjendens Magt saa stor i alle Retninger , at han ikke tillader Mennestebornene at betcenke dette , forend det er for sent , og Angeren allerede sorterer os over Ungdomssynder , som vi ikke mere tunne gjore ugjorte . Det gaaer som oftest saaledes med dette Bud , at man ikke betenker dets Vigtighed , forend man <b>selv</b> er blevet Fader eller <b>Moder</b> og har begyndt at hoste nogle bittre Frugter af sine egne Barndomssynder . — Io det er <b>sandt</b> , ogsaa et andet Tidspunkt pleier at vcekte os tll bittert at fole , hvad vi have forbrudt mod Fader og <b>Moder</b> nemlig en liden Stund efter , at de have udaandet deres sidste Suk r Jammerdalen en liden Stund efter , at det huldeste <b>Hjerte</b> mod os paa lordeu er standset , da vor eneste jordiske Fader eller <b>Moder</b> ikke mere findes ikke mere kan hore vor Bon om Forladelse ikke mere kan svare paa vort ommeste Sporgsmaal da man gjerne vilde midt om Natten grave dem op af Jorden for blot en eneste Stund at tale med dem . For var det maastee ikke saa magtpaaliggendc at underholde sig i barnlig <b>Kjerlighed</b> med en gammel Fader eller <b>Moder</b> , da den ungdommelige Lystighed behagede os bedre ; men nu , nu skulde Barnet gjerne tale med sin Fader eller <b>Moder</b> og gjore dem nogen Gloede ; nn ja nu er det for sent ; nu kan Dn isse en eneste Stund ajore det . O , at vi vare vise ! O , at vi itide besindede os ! Guds Ord tale saa sagte og mildt , men de lade sig ikke foragte ; man stal rette sig efter dem itide eller ogsaa bitterlig

<h4>793</h4>en hellig Nidkjcerhed og Iver for <b>Sandheden</b> og er dog igruuden intet andet end ct giftigt Had og Nag , som ikke kan udbryde eller bevise sig anderledes . Ligeledes ere og mange , ellers artige , anstcendigc , lcerde og forovrigt retskafne Mennesker , som gaa saaledes i hemmelig Vrede , Avind og Had og sorbittres deri , saa at de aldrig blive det vaer , og blive alle i den selvgjorte Bevidsthed , at de gjore det for <b>Sandhedens</b> og Retfærdighedens „ See derfor advarer Kristus saa alvorlig , at Alle og Enhver stal her see sig vel for , at han ikke bedrager sig med saadant Hykler : og falsk Skin : thi Ingen troer , hvor enfoldig denne Lcerc er , og dog træffer den saa store og fortræffelige Folk : thi med disfe Ord : Naar Du offrer Dm Gave paa Alteret , viser han jo klarlig , at han taler om dem , som tjene Gud og ville voere de rette Guds Born vg have den Berommelse , at de fremfor Alle ere den rette Blomst . Hvad fattes dem ? Intet uden det , at deres <b>Hjerte</b> imidlertid er fuldt af Had og Avind . Kjcere ! hvad betyder det , at Du uden Ophor beder , over Guds Ord , hengiver alle Dine Eiendele for Gnds Skyld , spceger Dit Legeme og gjor nok saamange smukke Gjerninger , men agter imidlertid ikke Guds Bud , hvilke han vil have holdte , og gjor Dig ingen Samvittighed over , at Du bestcemmcr og forsmceoer Folk , men vil dog vcere besynderlig hellig og give Gud store Offre ! Det er aldeles , som om Nogen havde stjaalet eller rovet en stor Sum Penge og siden giver en Almisse deraf for Guds Skyld . Saaledcs bedrage de Gud ( nei sig <b>selv</b> ) med det vattre Skin , som om han nu stuloe ansee dem for idel levende Helgene " . Saavidt Luther . Men her turde Nogen sporge : „ Er der da ingen hellig Brede , mgen ren Nidtjcerhed for <b>Sandhed</b> og Retfærdighed ? Stal hver Yttring i saadan Tone nsdvendia vcere af syndig Vrede , af Had og Avind ? " Svar : Der findes visselig en hellig Brede , en ren Nidkjcerhed for <b>Sandhed</b> og Retfærdighed , som Luther kalder „ en til Vrede optcendt <b>Kjcerlighcd</b> " , naar det nemlig er blot paa Noestens Vel og Guds Mre , Du har Dine Mue rettede , og Din Nidkjcerhcd blot vender sig mod det Syndige, Falske , Farlige , ikke mod Personen . Paa saadan hellig Brede have vi Exempler hos vor Herre Kristus <b>selv</b> og alle hans fornemste Tjenere . Og hvem skulde ellers kunne passe sit Embede som Loerer , Dommer , Fader , <b>Moder</b> etc. om der ikke var nogen hellig Nidkjcerhed , som iklceder sig Skikkelse af Vrede ? hver enkelt Kristen kan ogsaa optoendes af hellig Nidkjcerhed for Menneskers Vel og Guds Mre . Glem blot ikke , at Gud har <b>Oine</b> til at gjennemstue hele Dit <b>Hjerte</b> og hver dets mindste Bevarelse . Han seer det , han seer , om det virkelig er denne <b>Kjcrrlighedens</b> Nidkjcerhed for Noestens Vel , som driver Dig , og da alene en Nidtjcerhed mod dette Syndige og Skadelige , som Du seer , og ikke mod selve Personen . Vil Du vide , om det er

<h4>906</h4>man let om Andres Feil . Det er <b>sandt</b> , hvad en Lcrrer har anmccrtet . at „ det er et meget udmeerkende Tegn paa en <b>sand</b> Naade i Hjertet , at Mennesket har Tilboiclighcd til at anscc sin egen Synd for storst og mest utilborlig og derimod at finde Andres Synder undskyldelige og ringe " . Derfor , naar det tvertimod er saalcdes , at man ikte er meget besveerct , betynget og elendig over egne Synder , men maasiee seer sig <b>selv</b> nersten aldeles syndfri , men har en scerlig Tilboiclighed til at see og omtale Andres Synder , det er jo dog et ganske betænkeligt Tegn ftaa , at man horer til det Folk , som paa Kristi Tid sogte at straffe endog ham og hans Disciple for De talede med en saadan Nidkjcerhed for Herrens Lov , at Kristns for dem blev en Sabbatsbryder , en „ Fraadser og Vindranter , Tolderes og Synderes Ven " . Tamt , hvad er dog dette for en Aand ! det , som var „ storst i Loven , Dom , Barmhjertighed og Tro " , det lod de fare , men at Kristi Disciple ikke fastede som de , eller at deres Mester styrtede sine udmattede Krocftcr med Vintrccets Frngt , det udmalede disse til det Vcrrste . Og tamt , hvad det er for en Aand , naar man idelig soger at finde Feil og Synder ogsaa hos dem , som dog ingenting onstede sig hellere end Syndfrihed , naar man med Begjocrlighed opsnapper og paa det Vcerstc udtyder og udmaler deres virkelige eller formentlige Feil og soger paa den mest mulige Mande at nedscette og svcerte sin Nceste sor derved naturligvis <b>selv</b> at ansees for bedre og mere feilfri ! Kristus fceldte eu taaleligere Dom over de storste Syndere og Synderinder end over disse . Og denne vederstyggelige Synd bliver ikke ringere derved , at man tan forgylde sin Bagtalelse med vemodige Fagter , dybe Sukke og hjertegribende Klageudraab over den arme Nceste . Kristns har <b>Oine</b> som Ildsluer og gjennemstuer vel den fromme Skalk . Var der virkelig en saa om Bekymring for Noestens Sjcel , havde man virkelig en saa inderlig Deltagelse for ham , som man vil give det Skin af , saa sknlde man vel tale med Gud og med ham <b>selv</b> om hans Feil og hans Tilstand istedetfor idelig tun at tale derom med Andre og saa skulde man vel folge Kristi Forskrift om Maaden til at „ vinde sin Broder " . Just det , at man ikke solger Kristi Forskrift og den store Kjcrrlighedslov , vidner tilstrækkeligt om Vcrrdien af disse fromme Sukke og Udgydelser og viser , at de ere intet andet end en forgyldt Bagtalelse , udflydt af et nedrigt , skalkagtigt <b>Hjerte</b> , og at denne Nidkjcerhed er aldeles lig deres , der med deres hyklede Scedestrenghed fremforte den <b>Kvinde</b> , som var greben i Hor . Saaledes falde ogsaa disse endnu idag med deres Tunger over en Synder eller Synderinde og kunne ikke noksom drovtygge og atter drovtygge deres Urenhed , betragte og atter betragte oeres Synd og Daarlighed . Men Herren , som hender alle Menneskers Hjerter og Veie , han siger til saadanne : „ I Hyklere ! den iblandt Eder , som er uden Synd , han kaste den fsrstc Sten paa hende ! " Og for det Andet : regeres I af <b>Kjoerlighed</b> , saa sog

<h4>917</h4>lieden : men <b>Kjcerligheden</b> er Hjertets Sag . Her er Hemmeligheden i Uort Emne . Der ere mange Mennesker , lom ikte kunne bearibe hvoraf det kommer , at der aldrig bliver noget af , hvad de forescette fiq med Hensyn til <b>Kjcerlighedens</b> Bevisnmg , der er maen Kraft hos dem til de gode Forscetters Udfsrelfe , og paa den anden Side ere de altid paany faldne r famme fom de vestraffe hos fiq . De kunne virkelig bekymres derover og anstrcenge sia med at aflcegge denne Synd , men falde dog uophorlig deri . Hvoraf kommer det ? Svar : De give ikte Agt paa felve Kilden , men arbeide blot med Bcekkene . De kunne give Agt paa hundrede andre Sager oq tillige bevcege sig i mange vattre og hellige men give ilte Agt paa deres eget <b>Hjerte</b> , dets Ondstab Avind , falskhed m m . Og hvad er det da , fom fattes dem ? Det er zo altid den Helligaand , fom angriber Hjertet og ved ogsaa virker saadanne Krcefter , som ikte findes hos Naturen , arbeide uaa at udsve et Gode , som endnu ikke smdeS hos dem de ville vaaqe imod fyndige Ord og Gjcrninger , men give ikke Agt paa den bestandig fremtrængende Udfpringstilde , hvoraf Ordene udflyde, om Kriltus figer : Af Hjertets OverftsdrOed taler Munden Naar nu Ondstaben fordliver herskende i Hiertet , faa bliver ' det dels meget fvcert at vaage imod dens Udbrud i onde Ord oa Gierningcr , dels udrettes intet andet dermed , end at man nu bliver en HMer , at Ondskaben dolges for Mennester , men staaer lige vederstyggelig for Guds <b>Oine</b> . Om Guds Frygt loerte dem at tiende oq agte den fordommeligc Vederstyggelighed l Hjertet , oq de sogte en ' virkelig Forsoninq og Renselse i Blod da blev oqsaa Hjertet renset , da skutde ogsaa en ny inderlig Kicertighed oq Ydmyghed indtage den Plads , hvor kun Egenhcerlighed , Hovmod , Ondstab oq Avind for boede , og da stulde tillige den nye , hjertelige Kjcerlighcds og Ydmygheds Udtryk flyde over Tungen! Som for sagt , hvor Guds Aand boer og virker en tiltaqende Syndserkjendelse , Anger , Tro og Ydmyghed , der bliver det let at tcenke og tale vel om sin Nceste . da man mgen der kjender paa Jorden end sig <b>selv</b> , og naar man derhos har faaet Guds liflige Forsilkring om Syndernes Forladelfe for alle nne eqne fvcere ' te Synder , da bliver man deraf ikte blot inderlig bolet til gjerne at bcerc over med Andres , men af denne Made i fjertet flyde nu mange bellige Frugter og blandt disfe iscer Kjcer- Uqheden . „ Oq <b>Kjcerligheden</b> gjor Nceftcn intet Ondt " . „Kicerligheden er taalmodig oq mild , den bcerer ikke Avind den handler ikke falsk mod fin Noeste " . „ <b>Kjcerligheden</b> oMwses ikke , den <b>mor</b> intet Usommeligt , den soger ikke sit Eget , forbittres ike bcerer ikke Nag den glceder fiq ikke over Uretfærdighed , men glceder stg over <b>Sandhed</b> " . Kortelig , <b>Kjcerligheden</b> er Lovens Fylde . Gud foroae o ? . Troen og <b>Kjcerligheden</b> ! Gud indskrive alt mere i vort Sind <b>Kjcerlighedens</b> kongelige Lov : Hvad I Memnilene skutte ajore mod Eder , det gjsre ogsaa I mov nmen .

<h4>927</h4>det da ikke virre , at et Menneske bcerer Misundelse mod sin Nceste samt onde Lyster og Tragten efter hans Eiendom ! O , hvor forskrækkelig langt fra den rette <b>Kjcerlighed</b> maa itte det Menneske vcere kommet , fom ei alene ikke soger fin Nceftes Bedste , men rentud misunder ham hans Gode og ncerer Lyst til at faa det ! Det er denne grove , vederstyggelige Synd , Herren her forbyder . Og da forftaa vi tillige , hvad han derimod befaler os i dette Bud , nemlig at vi itte blot skulle vcere tilfredst med vor Lod og gjerne unde vorNceste hans Gode , men ogfaa at vi stulle fremme vor Nceftes Bedste og det ligefaa trolig og oprigtig fom vort eget . Derom siger Apostelen ci alene faaleoes : „ Eders Vandel vcere uden Gjerrighed ! Notes med det , I have " „Gudfrygt med Noifomhed er en stor Vinding " ( Hor . 13 , 5 ; 1 Tim . 6 , 6. ) men tillige faaleoes : „ Enhver feer ikke paa Sit , men Enhver ogfaa paa Andres Bedste " ( Fil . 2 , 4. ) . See , dette hedder at elske Nlrften fom sig <b>selv</b> : „ Enhver fee paa en Andens Bedste . " Det meft morke og arge <b>Hjerte</b> bnrdc dog oprores over Stjsnheden af et saadcmt " Bud : „ Enhver fee paa den Andens Bedste ; elsk Din Ncefte fom Dig felv . " Men mon det alene skal vcere et stjont Ord ? da Ingen virkelig gjore derefter ? Og hvem skal gjore derefter , om itte Dn og jeg , fom jo ogfaa ere Mennesker ? thi Gud har jo talet til ethvert Menneske . Eller har Gud Herren givet sine Ord uden nogen Mening , uden alvorlig at ville , at vi skulle gjore derefter ? Eller stulle vi gjcekke ham faaleoes , at vi blot prise hans Bud som stjonne , rette og hellige , men siden vende os derfra og aldrig begynde at gjore derefter ? „ Den Tjener , fom veed sin Herres Villie , men er bereder sig og gjor derefter , stal faa mange Hug " ( Luk . 12 , 47. ) . Men nu , om vi tvertimod vor onde Natur tjene vor Nceste i Alting og ikke forurette ham til en Halvskilling , faa er dette visfelig en ganske stjou Retfcerdigheo , men dog endnu kun for Mennesker . For Guds <b>Oine</b> staa endnn alle Dit <b>Hjertes</b> onde Begjceringer tilfyne , om Du end holder dem faaledes i Tomme , at Du itte udover dem i Vcerk og Handling . Du er udvortes from , men indvortes ugudelig , from i Gjerninger , men fuld af Forbrydelser i Hjertet , som dog skulde vcere selve Templet og Helligbommen for den hellige Gud . Naar Du i felvc Hjertet bcerer onde Tanker , Lyster og Begjceringer , saa er dette en Vederstyggelighed for Guds <b>Oine</b> , som han aldrig tan " igle . Derfor har han omsider ogfaa forbndt dette , naar han i det tiende Bud siger : Du skal ilte begjwre ld . e . ikke „ have Beajcer " til ) Din Nlrftes <b>Hustru</b> eller hans Tjener eller hans Plge eller hans Ore eller hans Asen eller noget af alt det , som horer Din Nlrste til . Her har nu den guddommelige Majestcet sluttelig udtalet, hvad det egmtlig er , han vil og mener i alle sine Bud , nemlig at vi stnlle vcrre helt rene og hellige , fom han felv er hellig ( 3 Mof . 11 , 44. ) . Her har den guddommelige Majestcet

<h4>933</h4>naar jeg har Dig . Du mit <b>Hjertes</b> Trost og Del . " Selve Helligheden skulle vi egentlig elske for Herrens Skyld ; thi vi stulle elske ham af vor ganske Styrke , af vort ganske <b>Hjerte</b> , vor ganske Sjel og vort ganske Sind . Ak . dette store Bud er jo dog ikte nogen Skjemt eller tom Ordleg , men Herrens evige Alvor ! Og hvem skal gjore derefter , om ikke Enhver , som sporger ester Gnd og Religion ? Hvortil skal det tjene at tenke ug tale i aandelige ' Ting . om man itke agter Gnds eget allerstorstc Bnd ? Derfor endvu engang : Hvem skal gjore derefter , om itke Dn , som horer og leser det . eller som vi for have sagt . stnlle vi saaledes drive Gjet med Gud . at vi prise hans Ord . men forglemme dets Efterlevelse eller udvelge visse Ord og Bud , uredens vi glemme det . som er storst i Loven? Gud oftvekte os fra en saadan forblindelse og Falskhed ! Om Du nu a , jor tvertimod dette Guds store Bud . ' saa at Du sidder hele Tuner , ja Dage med forfengelige Tanter og Begjeriuger til det Onde , maastee Bcgjer til Din Nestes Hnstru , maaskec nrene Lyster og Fantasier , nmaskee Bcgjer til Din Nestcs Eiendele eller ester egen Mre og Berommelse . medens Dit <b>Hjerte</b> burde uere opfyldt med Gud ' og hans <b>Kjerlighed</b> . O . hvor forfkretieligt seer det da ikte ud i ' Dit <b>Hjerte</b> ! Det er jo oa ret et Afgudshus , en Rovertule . Og saa kommer Du oisse Tider til Gud og oil ausees for from og hellig ! Tcenk endnn engang paa dette Forhold naar Du ei alene itke elsker Din Neste som Dia , <b>selv</b> og soger hans Bedste , saa trolig som Dit eczet . men tllertimod seer med Misundelse paa noget Fortrin . Gnd har givet ham enten i Mre eller Eiendom er da Dit <b>Hjerte</b> godt og Gud velbehageligt ? Eller naar Du har Begæring til Diu Nestes <b>Hustru</b> , uaar Du hengiver Dig til urene Forestillinger , begaaer Du da ikte Mgtestabsbrud ? Naar Du tenker paa nogen Uven med H » ad og Uforsonlighed , saa at et ondt Rygte om ham bramder Dig paa gjelen . ' om Du ei maa udtale ' det . er da Dit <b>Hjerte</b> godt og Gud velbehageligt ? Stil Dig med hele Dit Indre for den almægtige Guds <b>Oine</b> " og fol . at ' Du er en fordomt Synder ! Hans evige Alvor og Fordring er . at Du skal vare „ hellig , som han er hellig , " „ fuldkommen , som han er fuldkommen , " ja at Dit hele Vesen , Dit <b>Hjerte</b> , Dine Tanter og Begjeringer stnlle vere opfyldte af ham , hans <b>Kjerlighed</b> og hans Hellighed . Dette er Summen af Gnds forste og sidste Bud og af alle Budene tilsammen .

<h4>970</h4>Grad , som Aanocn er villig , hellig og Vaagen , i samme Maal fole vi ogsaa Fordaervelsen stcerk og paatroengeilde . Syndens Erkjendelse beroer blot paa , hvormegct Gud betyder sor Mennesket; har han en virkelig hellig og <b>sandfærdig</b> Gud for sig , da kan han fortceres , som alle Helliges Exemftcl viser . Eller findes der en eneste Kristen saa hellig og alvorlig , at han endog kun cn eneste Time opfylder , blot hvad det storste og ypperste Bud fordrer ? Vi sporge dem , som ere mest alvorlige og have mest af Guds Aand og Kraft . Men hvad er al Fromhed , om Du ikke holder det storste og ypperste Bud ? Det Bud fordrer , at Du skal elske Herren Din Gno af Tit ganske <b>Hjerte</b> af Din ganske SM , af Din ganske Styrke , af Dit ganske Sind , og derhos , at Du stal saa helt og aldeles forlade Dig paa ham alene og frygte ham , som hans guddommelige Magt og Trofasthed fortjener. Er det ikke <b>sandt</b> , at om Dn saaledes elsker Gud af Dit ganske <b>Hjerte</b> og forlader Dig paa ham alene , saa at Gud alene er det hoieste Formaal for Din <b>Kjwrlighed</b> , Din Tillid og Frygt , saa maa Din Sjcel vcrre i en evig Ro , i den uforanderlige Nydelse af dette eneste Gode , Dn begja ' rer , den uforgængelig / Gud ? Da maa Du tro , at ikke et Haar falder af Dit Hoved uden Din himmelske Faders Villie , at ikke det mindste kan hamdes Dig , ikke et Ord , ikke et <b>Blik</b> saare Dig uden Din himmeNe Faders Villie . Naar Dn da intet andet elsker end ham og hans Velbehag, saa maa jo deraf folge , at Du alletider er lige rolig og lykkelig , hvad der end vederfares Dig , blot fordi Du vced , at Alt kommer fra Din Gud , og Du elsker ingeuting andet end ham og hans Velbehag . Hvor er nn det hellige Menneske , som holder dette Bud ? Vi ville nu tale med en saadan . Er Du lige rolig og lykkelig , om Nogen tager al Din Eiendom fra Dig og Du nedsyuker i Fattigdom og Mangel ? lige rolig og lykkelig , om Nogen derover D ' ig Dit gode Navn og Rygte , om Du bliver cerelos , foragtet og afskyet af alle Mennesker for hele Dit efterfolgende Liv ? lige rolig og lykkelig , om en svcer Sygdom , ja en Morder forkorter Dit Liv ? Om det er <b>sandt</b> , at Du elsker Gud af Dit ganske <b>Hjerte</b> , af Dit ganske Sind og af alle Dine Kroefter og derhos troer paa ham alene , at ingeuting <b>moder</b> Dig uden hans Villie , saa maa Du nodvendia . vcere lige rolig og lykkelig ved alle disse Men maastee det er saalangt derfra , at Dn tvertimod bliver fornroliget over et ganske ringe Tab ! at om Dn blot faaer at vide , hvorledes man har talet ilde om Dig eller blottet nogen af Dine Svagheder , det forstyrrer Din Ro for hele Timer og Dage ? Ja maastee blot et foragteligt <b>Blik</b> foruroliger Dig ! Hvorledes elsker Dn da Gud alene og hans Velbehag ? Ja foler Du , at Du rigtig varmt elsker Din Gud , saa at Dine Tanker ere bestandig hos ham ? maastee Du tvertimod meget livligere elsker og tamter paa noget Menneske ! Men videre : Er det ikke <b>sandt</b> , at vi , som ere gjensoste med Guds Sons Blod fra alle vore Synder , fra Dsden og fra Djcevelens

<h4>972</h4>Vold , burde intet hoiere Maal have for vort Liv end at forherlige ham , som er dod og opstanden for os ? Er dette nu virkelig til alle Tider DinStreeben ? Er det ikke <b>sandt</b> , at om Du elsker Gud af Dit ganske <b>Hjerte</b> , da skal Du ikke kunne trives saaledes ved nogen Ting som i fortrolige Bonner og Samtaler med Din Gud ? Har Du det virkelig saaledes , at Du bestandig vil omgaaes med ham i Bonnen ? Maastee det er saalangt derfra , at Du hellere forretter adskillige Hussyslcr end omgaaes med Gud i Bon ! Ak , hvorledes er det ' da med Dit Forhold til det forftc og storste Bud ! Derncest skulde Du ogsaa elske Din Na ' ste som Dig <b>selv</b> . Tamt altid alvorligt vaa de storste Bud af Hcrreu Din Gud . Hoad er al Fromhed , om vi ikte forst holde de vigtigste Bnd ? Er det virkelig <b>Sandhed</b> , at Du har ligesaa megen Omsorg for Din Ncrstes Bedste som for Dit eget ? at med Din „ Nceste " menes ikte blot en og anden Ven , men alle Mennesker , Venner og Uvenner . Er Du ligesaa ivrig for ethvert andet Menneskes Vinding som sor Din egen ? ligesaa om for et dadlende Ord om Din Nceste , som naar Du horer , at man har talet ilde om Diss ? Videre : Du troer , at Enhver , som doer uomvendt, bliver evig fordomt . Dersom Du da elsker Din Nceste som Dig <b>selv</b> , saa maa Du anvende samme Flid , Iver og Nidkjcerhed for ethvert andet Menneskes Omvendelse , som om det gjaldt Din egen Salighed eller Fordommelse . Du umager Dig for nogle Faa ; men maastee Du daglig seer mange Uomvendte , for hvis Vcekkelse Du ikke gjor Dig mindste Mme ! ' Hvorledes er det da med Din <b>Kjcerlighed</b> til Ncesten ? Og hvorledes seer det ud med alle de ovrige Buds Efterlevelse ? Er det ikke <b>sandt</b> , at mange vanhellige Ting rore sig i Dit <b>Hjerte</b> ved en ganske ringe Anledning , Ting , som Gud Herren hader og fordommer , enten Vrede , Avind . Had eller Hoffcerdighcd og Egenkjcerlighed eller urene Lyster eller Begjcerlighed efter Andres Eiendom m . m . Og vi tale dog endnu om de troende Kristne , som erc vaagne og tjende deres Synd . Der sindes intet Folk , som jamrer og beklager sig saameget over Synden som de Hellige . Hvorledes gik det da , om Gud skulde domme os efter sin Lov ? Maa ikte de Hellige bede : „ Gaa ikke til Doms med os ; thi for Dig er ingen Le- Vende retfoerdig ! " Saaledes vidner Skriften : „ Iblandt hans Hellige er ingen ustraffelig . " „ Hvad er et Menneske , at det skulde voerc rent , eller at den skulde voere retfoerdig . som er fodt af en <b>Kvinde</b> ? Himlene ere ikte rene for hans Oi ' ne ; hvormeget mere er Mennesket vederstyggeligt og stinkende for ham og drikker Uretfærdighed som Vand " ( Job . 15 ) . Og atter siger Skriften : „ Der er retfoerdig , end ikte een . Alle ere afvegne , de ere tilsammen blevne uduelige , der er Ingen , som gjor godt , der er end ikte een " ( Rom . 3 ) . Hver Hellig behover daglig Syndernes Forladelse . Hvorledes skulle det da gaa os , dersom vi endnu skulde dommes efter Loven .

<h4>974</h4>Det er meget vigtigt , at vi af Ordet vel kjende og tro det dybe Syndefald og Fordoervelsen i al Menneskenatur , ogsaa i dets Levninger hos de Hellige , ei alene for at vi maatte vel betcenke, vurdere og lovprise den uudsigelige Naade , at vi ere fri fra Lovens Domme , men ogsaa for at Samvittigheden maa kunne forblive i denne Frihed . Thi om vi ikke af Ordet ere vel underrettede om denne Fordærvelse , saa stulle vi med al Kundskab om Kristi Fortjeneste dog aldeles nedgraves i Elendigheden og fortvivle , saasnart vi erfare den hos os <b>selv</b> . Thi om jeg , som troer paa Kristus , sinder en Syndeelendighed hos mig , som jeg ellers troer , kun alene kan sindes hos de Vantro og aldrig hos Guds Born , saa hjcelper al Kristi Fortjeneste mig ikke ; thi det hedder nu : Du har hverken Del eller Lod i dette Ord ; thi Dit <b>Hjerte</b> er ikke ret for Gud ( Apg . 8 , 21. ) . Derfor maa vi endnu noget noiere overveie dette Punkt . Vi saa i Skriftens Vidnesbyrd , at ogsaa „ iblandt Guds Hellige er ingen ustraffelig , " at „ den er ikte til , som er retfærdig ( i sin egen Person ) ikke een ; " men saa tcrnker ofte en enfoldig troende Sjcrl , som scedvanlig bliver anfcegtet med alle Modsigelser af Hjertet og Satan : Ja Ingen er ustraffelig , Ingen er retfcerdig for Guds <b>Oine</b> , for hvem ikke engang Himlene ere rene , Guds <b>Oine</b> , som see det , der ikke er ; men aldrig kunne de sande Hellige have og fole saadan virkelig Synd i deres Hjerter, som f . Ex . syndige Lyster og Begæringer , Koldsindighed og Mistro mod Gud , Hovmodstanker o . dcsl . thi Helliggjsrelse , den Helligaands Vocrk , ja Navnet „ Hellige " maa ogsaa betyde noget . Naar jeg nu fsler saadant hos mig , som er virkelig Synd , ja det udbryder i Ord og Gjerninger , ' saa kan jeg jo umulig have nogen Del og Lod i Kristus ' og hans Fortjeneste . " Og paa denne Maade skeer det , blot ved Misforstand om Helgenenes Hellighed og ved Ukyndighed om den almindelige Fordærvelse , at de som virkelig kjende dcrcs Synd , de gaa , med alt det Evangelium de hore , dog i en hemmelig Selvfordømmelse og savne den Tro og Fortrostning , som skulde give dem <b>Kjcerlighcd</b> , Liv , Lyst oa. , Fraft til det Gode , og tun de troste sig , som ere sovende nok til ikke at mcerke Synden hos sig . See , det er formedelst denne Tankeforvirring , det bliver saa nodvendigt ogsaa at betragte saadanne Steder i Skriften , som vise os med levende Exempler , hvad der menes dermed , at „ iblandt de Hellige er ingen ustraffelig . " Nu , <b>sandt</b> siger Du , at Helliggiorelse og deu Helligaands Vcerk ogsaa betyde noget . De , der have en saadan Tro , som ingen Helliggjsrelse virker , ere ikke rette Kristne . „ Den , som ikte har Kristi Aand , den er ikke hans " ( Rom . 8 , 9. ) . Men Dm Misforstaaelse kommer deraf , at Du ikke betcenter , at der ere to Naturer hos de Hellige , det gamle og det nye Menneske , Kjsdet og Aanden . „ Det , som er fodt af Vlanoen i os , det er Aand ; men det , som er fodt af Kjodet , er Kjod " og har sin adamitiste Art , saalcrnge som et Stykke deraf er tilbage , d . e . saalcrnge vi

<h4>986</h4>meegtig , at de ere noerved aldeles at fortvivle ; de fole en Koldsindighed for Gud en Vantro , Sikkerhed og Haardhed , som ere aldeles forstrcekkelige ; intet Liv i Bonnen , ingen Smag og Kraft i Ordet , ingen hellig Nidkjoerhed for Guds Mre , heller ikke nogen almindelig <b>Kjcerlighed</b> til Ncesten , men blot Dod og Afmagt i alle Ting . Derncest rore ogsaa mange uvoerdige Ting sig i deres <b>Hjerte</b> , saasom Maalmodighed , Vrede , Avind , urene Lyster og Begjceringer, Egennytte , Hovmod m . m . Hvor skulde de med al oenne Syndeelendighed ' kunde vcere Guds elskelige Born og Arvinger, om de skulde betragtes og dommes efter Loven ? Og hvorledes skulde de kunde have en rigtig Hjertets Fred med deres Gud , og Vished om hans Velbehag ' og Venskab , om de havde den Mening , at han skulde domme dem efter sin Lov ? See da , hvor aldeles nodvendigt det er , at vi ere friede fra Loven , om vi skulle blive salige og om vi skulle have Fred med Gud i vore fjerter ! Men Samvittighedens Frihed fra Loven er ogfaa noovendig for den sande Helltggjorelse , og for at kunne gjore virkelig gode Gjerninger . O , at Gud aabnede Manges Hjerter for ' denne <b>Sandhed</b> , som for den blinde Fornuft er en ren Galenskab ! Apostelen siger udtrykkelig : Ligesaa ere og I , mine Brodre , dsde fra Loven paa det I skulle boere ' Gud Frugt . Og videre: Men nu ere vi som Dode loste fra Loven , under hvilken vi holdtes , saa at vi skulle tjene i Aandens nye og ikke i Bogstavens gamle V « esen . ( Rom . 7 , 4. 6. ) . Og paa samme Maade taler Apostelen i Gal . 2 , 19 : „ Jeg er formedelst Loven dod fra Loven , paa det at jeg skal leve Gud . " Moerk dog , at Apostelen hermed udtrykkelig erklcerer , at mau ikke kan leve Gud , ine boere Gud Frugt , uden man forst er dod fra Loven . Betragt her blot tvende Forhold forst det som Erfaringen lerrer : Saalcenge en Sjoel ikke er frigjort fra Troeldomsaanden , men arbeider paa fin Fromhed med <b>Oiet</b> paa sig <b>selv</b> og paa Loven og seer Gud som den , der fordrer Noget af ham , saa er Alting dodt , koldt , tungt og modbydeligt i Hjertet , og med alle Forscrtter bliver der ingen Kraft , ingen Udforelser , blot Tanker og Ord og nogle udvortes lagttagelser , som ikke ere for svoere for Naturen . Videre , om Loven ogsaa angriber Hjertet og den indre Fordoervelsc, saa bliver denne oprort og opegget til dobbelt Kraft , som Apostelen siger : Da Budet kom , blev Synden levende og virkede al Begærlighed i mig m . m . O , hvormange amgstclige Trcelle sukke ikke under denne Erfaring og kunne ikke forståa , hvori Feilen ligger ! De hore og lces ' e Guds Ord , de raabc og bede , de gllll til Skrifte , og Alt er forfcengeligt , Alt er idel Dod og Afmagt; kun Synden er levende og moegtig . O , at de vilde boie deres Ore til Guds Visdom , „ da skulde deres Fred blive som Vandstromme og deres Retfærdighed som Havets Bolger , " da skulde de saa en ny , himmelsk Lyst og Kraft i deres Hjerter ! Feilen er , at Du aldrig er bleven ret frigjort fra Tra ' ldomsaaget , aldrig ret har smagt , hvad den sande Tro vil sige , aldrig er bleven

<h4>988</h4>ret salig blot i fri , uforskyldt Naade . Om Du fil det Lys , som ikke en Broder kan give den anden , men som er cn Guds Aabeutmrelse i Sjoelcn , en himmelsk Gave , det Lys , som forklarede lesum saaledes , at Dn saa denne uudsigelige Naade , hvorved den himmelske Fader , under hvis Bud Du trcrllcr , har saaledes forbarmet sig over os , at hau har hengivet sin Son til at opfylde for OS , alt hvad Loven nogensinde trcever af os , at han ogsaa for Dig allerede har udfort alt , hvad Tu nu arbeider , Paa , og at Gnd nu intet andet vil med siu Lov end udmatte og drcebc Tin Selvretfa-rdighed , efterdi han aldrig har teenkt , at Du formaner at gjore andet end fynde ( Rom . 3 , 12 — 19 ) men at han er forsonet for al Din Uretfærdighed , saadan som Du er , elsker Dig , sust som Tu er , har <b>selv</b> borttaget alle Diuc Syuder og tegnet Dig i Himlen , just som Du er , for Kristi , Tin Broders og MidlVrs Skyld . Om Du i Aandcns Lys saa alt dette og saa , hvorledes den himmelske Fader aldrig gjor Regning vaa Dig men har Alt og nok i sin elskelige Son , ' for hvis Skyld han aldrig , aldrig seer vaa Dine Synder , ja , om Du saa , hvorledes Du midt under Din dybeste Nedslagenhed over Synden er tcrkkelig og elskelig for Gnds <b>Oine</b> i Kristus , saadan som Du er om Du i Aandens Lys saa dette , sandelig , Du skulde blot groede af <b>Kjcrrlighed</b> og springe af Gloede , og alt , hvad der hidtil har vcorct Dig altfor svcert og umuligt , skulde blive Dig let , lifligt og elskeligt . Hvad var det , som gjorde , at Martyrerne gik med Fryd til Baalet , at Wineentius kaldte de glodeude Kul moser , da han stegtes , at svage <b>Kvinder</b> sagde , at de aldrig havde gaaet til deres Dandse med saadan Glcrde ' fom nu til Baalet ? Ingenting andet , end at de vare drukne af den usigelig store Naade , at Guds Son var deres Broder , og at de ved ham vare Guds Venner og hans Oincs Lyst . Sce , dette er , hvad Skriften saa udtrykkelig siger : Fryd i Herren slal tmre Eders Styrke . Og : Naar Du troster nut <b>Hjerte</b> , saa lober jeq Tine Budords Vei .

<h4>991</h4>derens Ris eller lokket dertil ved et Lofte , da det jo burde bevise denne Venlighed af sit eget <b>Hjertes</b> Folelse for sin <b>Moder</b> . Nu er det af alt Guds Ord aabenbart , at Gud Herren ikke vil have nogen tvungen Tjeneste ; han vil egentlig have Hjertet , dets <b>Kjoerlighed</b> og Lyst . Ligesom en Mgtemand , der rigtig elsker sin <b>Hustru</b> , ikte er tilfreds med blot at faa Tjenester af hende , men vil allerforst eie hendes <b>Kjoerlighed</b> , og om han opdager , at hun blot for hans Eiendoms Skyld anstiller sig , som hnn elskede ham , men ikke i <b>Sandhed</b> gjor det , da maa jo al hendes Venlighed blive ham voemmelig og knn opvække hans Vrede just saaledes er ogsaa Gud sindet . For hans Sine er al den Fromhed og Gudfrygtighed vcemmelig og forkastelig , som oves blot fordi man slal , man bor , man maa saalcdes , d . e . Oves blot af lovmæssig Bevæggrund og ikte flyder af en frigiort , salig og taknemmelig Aan ' o , af Brudens <b>Kjoerlighed</b> , som siger saaledes : „ Nu gjor jeg ikke saameget som at optage et Straa for at fortjene Saligheden og Gnds Venskab eller forsone min Synd ; thi alt dette har min Blodbrndgom gjort , da han i tredive Aar trallede for mig under Loven og siden led Korsets blodige Dod ; men det Lidet , jeg gjor , det gjor jeg alt , fordi jeg ingenting behover at gjore for min Benaadning , just fordi han har gjort Alt for mig og er <b>selv</b> til enhver Lid min Retfærdighed for — alene fordi , jeg nn er i et saadant Naaderige , at alle mine Synder aldrig tilregnes mig , at jeg aldrig mere stal dommes efter Loven . Just denne Frihed bevoeger mig saa inderligt til gierne at gjore og lide alt , hvad min Gnd vil , efterdi han har taget mine toerende Hjertesorger over Synden oa Loven bort fra ung . Ja da han har taget den store og tunge Byrde , min Salighedssag, ftaa sig , da vil jeg saa gjerne boere lidt paa disse lettere Byrder , at forsage Verden og Synden , underkue mit Kjod og tjene min Noeste i <b>Kjoerlighed</b> med Gjerninger , Ord og Taalmodighed , give den Hnngrige ' Brod , den Husvilde Herberg , besogc nogen Syg eller Fange , forlade nogen Uven og have Taalmodighed med et eller andet besooerligt Menneske eller et bcsvcerligt Kald i Livet . Efterdi min Herre ogsaa har voeret saa naadig at ville lade saadanne Smaatmg behage sig , saa er jeg noesten ligesom halv i Paradis over , at jeg maa og kan gjore den trofaste , dyrebare Herre nogen behagelig Tjeneste . Alt dette er mig saa hjertelig kjcert , fordi han var saa overmande god og indrettede dette Naaderige, at ingen ' Hynd tilregnes mig , at jeg aldrig stal dommes efter Loven " . Saadanne ere de rette Beoocggrunde til det Gode . Saadant betegner ogsaa det Ord af Apostelen : Kristi <b>Kjcrrlighed</b> tvinger os , idet vi domme saaledes , at dersom Een er dos for Alle , saa ere de alle dode s 2. Kor . 5 , 14. ) og saadant mente Apostelen, da ban sagde : „ Ligesom ere og I dode fra Loven , paa det I skulle ' boere Gud Frugt " . Du faacr visselig Din Lon her paa Jorden ogsaa for det

<h4>993</h4>Gode , Du qjor af lovmcessig Bevæggrund , det er visselig ogM nyttigt for Dine Naboer og Dit Husfolk ; men den sande Hellwqj'orelse er det ikke eller virkelig gode og Gud velbehagelige G ' jerninaer , som han vil omtale paa den yderste Dag . Men er der kun " Svorgsmaal om Saligheden , saa kan Du gjerne jpare Din Moie med det Gode og give Dig til at leve frit i Synden Dn taber ingenting derved ; Du bliver dog ikte salig ved en saadan hellighed , som flyder af lovmcessig Bevceggrund : . thi saamanqe , som holde sig til Lovens Gjerninger , ere under forbandelse' ( Gal . 3 , 10. ) . Kun saameget Godt , Du <b>mor</b> , M ' tn Du er saa uforskyldt benaadet og er fri fra Loven , tun faameqen <b>sand</b> Helliggjorelse og gode Gjerninger har Du . Mceri og mindes dette vel . og skriv det uudsletteligt i Dit Hind : kun saameget, som flyder af Naaden , af Tro og Kærlighed , as Din Frihed fra Loven , kun saamegen <b>sand</b> Hclliggjorelse har Du . Men her turde Nogen sporgc : „ Stal da Loven tzene til ingenting for en Kristen i hans Helliggjorelse og er da en Kristen altid saa oplivet af Troens Mcede og Kærlighed , at Han aldrig gjor det Gode af lovmcessig Aarsag ? " . Forst er Loven altid til en uundvcerlig Tjeneste og Velsignelfe i Helligqjorelsen ved den Rettesnor og det Lys , vi have i dens Bud ; og derncrst bliver denne Rettesnor os altid til Tugt , til en nodvendiq oq heldbringende Tugt og Sondcrwuselse for vort underfundige fjerte , som ellers snart kunde falde i Hovmod , Sikkerhed , Letsindighed og lignende Vederstyggeligheder . Men selve Bevcegarunden oq Hjertegrunden i en Kristens Helliggjorelse maa vcere den store Naade . Dette stal Du forftaa paa folgende Maade : Det er aldeles <b>sandt</b> , at Naadelyset i Troen ikke altid brcender lige stjont og levende ; Troen , Glcrden og <b>Kjcerligheden</b> svcekkes ofte under Vandringen , og hvor alvorligt jeg end arbeider dermed, saa kan jeg ikke naarsomhelst tage mig en Mdan brcendcnde Tro , som jeg vilde . Men derpaa beroer Sagen . Om jeg luot har den Tillid til min Gud ved Kristus , at jeg dog tan tale med ham som hans Born , kan tro selve Naadestanden , om end med en mere tor Tro , som dog holder sig fast ved Herrens Tilsigelser, saa virker dog denne Tro en villig Acmd , saa at jeg gjerne vil tvinge mit Kjod efter hans Budord . Jeg foler det vel ikke saa let som i de mere glade Tider , naar han fryder nnt <b>Hjerte</b> nei , der bliver efter Folelsen ofte en lovagtig Tyngde i min Vandring , saa at jeg mere foler Loven i mit Indre end Euangeli Naade og Liflighed . Men Forskjellen fra dem , som „ holde nq til Lovens Gjerninger , " beftaaer deri , at medens disse gjore det med Forscet og Mening Mcerk : de have rentud den Memna , at vor Naadestand og vor Helliggjorelse stal bero paa vort Arbeide i Gudfrygtighed , og ere saaledes emge med W <b>selv</b> i denne Strceben , saa at vor Vceren og Gjoren er rentuo deres Psalme og Vise , deres forste og sidste , saa er derimod en Troende , ogsaa under sine lovbetyngede Tider , dog en Henoe af

<h4>995</h4>sin egen Selvbetydenhed , saasnart han marker den : han straffer og misbilliger den og giver Gud Ret i hans Ord , at i os er idel Synd og Afmagt og i Kristo alene baade vor Retfærdighed og Styrte . Han jager efter blot at kunne gribe og tjcndc Kristum og hans Opstandelses Kraft og vced , at da blev Alting hjulpet og helbredet , men lider kun af Svaghed . Hans Sind og Forseet er saaledes evangelist , det er blot Folclsen og Naturen , der ere lovagtige . Og skjondt han da maa ligesom ' tvinge sig til meget nnder Folelsen af Kjodets Modstand , saa er det dog den store Naade , som han troer paa , der bevager ham til saaledes at ville tvinge sit gjenstridige Kjod . Han elsker Herrens Bud , eudog hans Trusler , ja hans ' Ris og Tugtelse , blot fordi de bidrage til Kjodets Dodelse : men just ' denne <b>Kjarlighed</b> til Loven og Truslerne har han deraf , at han troer den store , uforskyldte Naade , og saaledes ere jo da hans Hjertegrund og Bevæggrund evangeliske . Helt anderledes er det , om man bliver „ fangen nnder Traldommens Aag , " naar Tankerne forst og sid ' t staa paa os <b>selv</b> , og hvad vi ' skulle vare og gjore , hvorved Barnetilliden kvceles , " og Loven bliver rentud den regerende Magt i Samvittigheden . Da indfader atter det Syndevocsen og den Asmagt , som Loven altid virker . Og heraf see vi da , hvor nodvcndig Samvittighedens Frihed fra Loven er ogsaa sor den sande Helliggjorelse . O , at hver iiristen derfor i alle sine Levedage tog Apostelens Formaning vel til <b>Hjerte</b> : Bliver stadige i den Frihed , hvormed Kristus frigjorde os , og lader Eder ikke atter tvinge under Tnrldoms ' Aag ! lGal . 5. ) . Saa siger og vor Herre stristns <b>selv</b> om denne Sag : Bliver i min Kjcerlighed! Bliver i mig ! Ligesom Grenen ikte kan bwre Frugt af sig <b>selv</b> , uden den bliver i Trcect , saaledes kunne I ikte heller , uden I blive i mig . „ Ja , " siger Du , „ jeg har vel sect herlige Ord og Beviser for den store trostcrigc <b>Sandhed</b> , at vi , som ere fortabte for Loven og holde os til Kristum , stulle ikke dommes efter Loven , men vare saa fri fra dens Domme , som <b>Kvinden</b> er sri fra Mandens Lov , naar denne er dod oa , begravcn : jeg har ogsaa seet , hvor aldeles nodvcndig denne Frihed er , ikke blot for at kunne forblive i Guds Naade , men ogsaa for den sande Hclliggjorclse ; men det er mig dog aldeles umuligt at kunne rigtig tro og fastholde denne Frihed ' for min egen Del ; thi dels ' er det ' altfor stort og hcrliat , at jeg arme Synder stal leve i en saa bestandig og urokkelig Naade hos Gud , som om ingen Lov eller Synd var til ; og dels foler jeg . ikke det mindste af en faadan Naade og Frihed , men foler tvertimod det modsatte , nemlig at jeg endnu idelig anklages og dommes af Loven . Hvorledes kan ' jeg da tro , at jeg er aldeles fri fra alle Lovens Domme , at Gud elsker mig og ansccr mig , som om jeg aldrig syndede , som om ingen Lov og derfor ingen Synd var til , men idel Renhed , Retfærdighed og Velbehag for hans <b>Oine</b> og alt dette blot for

<h4>1107</h4>Omsorger ; Menneskene og Dyrene have havt falles Fornødenheder Religionen har blot hort til Templet og Sabbaten og nu opstaaer paa engang en Sporgen efter <b>Sandhed</b> , efter den rette Vei til Himlen ; nu begynder Guds Ord at blive fremtaget oa. blive Gjenstand for Granskning og Samtale , og gamle forstandige Mennesker , som i al deres Tid have levet i fuldkommen Ro for Sporgsmaal om Sjcelen , begynde nu at blive urolige for denne , saa at nogle Foedre og Loercre blive forundrede , bestyrtede og urolige over disse „ Nyheder " . Visselig er dette en mcerkvcerdig Tildragelse ! Men hvad er nu det egentlige Forhold , naar saadant viser sig , hvad er det , sum da skeer ? siger Apostelen i vort Tcxt naar han har beskrevet den aandeligc Dod : Men Gud , som er riss paa Barmhjertighed formedelst sin store <b>Kjwrlighed</b> , hvormed han elskede os , opvakte os , da vi vare dode i Overtrcrdelser Det er Gud , siger Apostelen , som gjor denne Bevcegclse. det er en „ DPVwklelse , " siger han , det er Guds Rige , som tommer til den Egn , den Menighed , det Hus , den Sjcrl , hvor dette Nye opstaaer . For sin store Kj-rrligheds Skyld , hvormed han har elsket os , gjor han os levende . ' For Guds Oinc vare vi saa indesluttede i Kristus , da han dode for os og atter blev levende , at derom ogsaa siges : Om Een er dod for Alle , saa ere de alle dode i ham ( 2 Kor . 5 , 14. ) om Ven er opstanden for Alle , saa ere de alle i ham , „ tilligemed ham " opstandne - for Guds <b>Oine</b> , efter Tilegnelfesloven ; men dette maa ogsaa skee i os , blive en Virkelighed for os , saa at vi ogsaa ved Aanden opooekkcs tilligemed Kristus ; vi maa blive „ eet Legeme og een Aand med ham . " Det er dette , der skeer , naar Herren opvwkker Hjcclcne fra den aandclige Dod og indsætter dem i det himmelske Va ' sen i Kristo ; de blive fodte paany , „ skabte i Kristo Jesu til gode Gjerninger " ( v. 10 ) . See nn her , hvilken Forandring ! Den , som forud var faa dod , at han aldeles ikke frygtede i alle sine Synder , faaer nn et <b>Hjerte</b> , som frygter for alt Ondt , ogsaa det han ikke veed med sig ; han frygter saaledes for sit falske <b>Hjerte</b> og Fjendens Forblindelse , at han ikke engang vover at tro sin egen Bevistheo , men maa raabc : Ransag mig Gud . og prov mit <b>Hjerte</b> , tjeno om jeg er paa en uret Vei etc. Den , som for i faa mange Aar havde idel jordiske Anliggender , Omsorger og Bestræbelser , ' for ham har nu aabnct sig en hel Verden af nye Formaal , aldeles nye Omsorger , nye Udsigter og Bestræbelser , nye Sorger og nye Glceder . Kilden til sine storste sorger og Bekymringer har han nn faaet i sit eget <b>Hjerte</b> , i sin Syndefordaervelse , og Formaalet for al sin Strceben i Kristus og hans Naade . Og den , som forud ikke kunde tro , at Ordet „ Aryd i Herren " havde nogen virkelig Betydning , at noget Menneske paa Jorden kunde have nogen virkelig Glcrde i Gud , faaer nu <b>selv</b> en saadan Erfaring deraf , at al Verdens Gloede synes ham som en Jammer og Vederstyggelighed imod

<h4>1136</h4>forst cr udtroettet af min egen Villies Ondstab og siden saa gjennemtrcrngt , dels af Guds Storhed , at hans Villie er mig mere end hele min Salighed , dels af hans <b>Kjcerlighed</b> og Liflighed, saa at , naar blot hans Villie steer , er Alting vel , da har jeg et faadant <b>Hjerte</b> , fom virkelig elsker Gnds Villie . Her maa nn Enhocr give Agt paa fin Tilstand . Vi have her atter et Stytte , som « åbenbarer vort <b>Hjertes</b> Inderste . Vi erindre atter om , at Bonnen maa vcrre et Udtryk as Menneskets egen Omsorg , Begjwring og Anliggende . Ikte hvemsomhelst stal da kunde bede denne Bon . Vi tale dog ikte om den store Hob , som lever aldeles frit „ efter Kjodets Begjcrringer ; " men der er endda to Slags Mennesker , som ikke ret kunne bede denne Bon . Forst er der Nogle , som visselig bede den ret alvorligt og flittigt , efterdi hele deres Salighedshaab i sin inderste Grund beroer paa deres Opfyldelse af Guds Villie disse „ Tjenestekvindens Sonner " , hvoraf nogle med tung og moiefuld Kamp sutte efter Naade til at kunne gjore Guds Villie , medens derimod andre iblandt dem , just ved Tilfredsheden med deres Gjoren og Beden , have en vis Lyft dertil , ja en vis Tro og <b>Kjoerlighed</b> , men altsammen grundet just paa denne deres egen Fromhed . Da de dog aldrig ere „ ved Loven dodc fra Loven , " aldrig ere blevne helt tilskamme , fortabte og fordomte i sig <b>selv</b> eller ere blevne salige ved Kristi Gjerning , Bon og Lidelse aleue , saa hore de endnu til „ Tjenestekvindens Sonner , " som ikte for Guds Skyld , af Kristi <b>Kjcrrlighed</b> , men for deres egen Skyld , deres egen Fromhed og Salighed jage efter at gjore Guds Villie . Og „Tjenestekvindens Son stal ikke arve med den fri <b>Kvindes</b> " ( Gal . 4 , 22 — 31. ) . Men der er ogsaa en Slags evangeliske Folk , som ikke heller kan bede denne Bon disse Evangeli lose Tilhængere, som gjerne hore det , der lyder lifligt og vel , samt have en meget uforstyrret og jevn Tro . men en saadan Tro , som ikte medforer noget helligt Sind , saa at det angriber deres gamle Menneske og gjor , at Guds Villie bliver deres Levnetslov , bliver Formaal for deres Hjertevenner , men de leve aldeles fri for denne Omsorg for Guds Villie ; de have i deres Liv nogen Form af Kristendommen til et vist bestemt Maal , med hvilket de noies , og derfor ere de siden fri fra denne Omsorg . De lcese Bonnen : Slee Din Villie ! med eller uden Tanke , men om end Tanken folger med Ordene , er det dog ikke mere , end at det bliver en blot Tanteosn . ikke Hjertets eget Anliggende . Du . som lceser dette , stands dog en Stund for Guds Ansigt , for de <b>Oine</b> , som skue <b>Hjerte</b> . Tanke og Forsoet . Hvorledes er det med Dig i dette Stykke ? Du kan vel vide mZd Dig <b>selv</b> , om Du pleier at have saadan Omsorg , omgaaes med saadanne Sukke og Bsnner : „ Gud , viis mig Din Villie , hjcelp mig at gjore Din Villie ! " Det er jo umuligt , at den Helligaand kan bo i et <b>Hjerte</b> uden at opvække saadan Omsorg derinde . En Kristen kan vel ofte overfare Bonnen : Slee Din Villie ! saa adspredt , at han ikke engang

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015062608031'target=_>, 1867, Bibelen, eller Den hellige Skrift</a><h4>4592</h4>Veien og <b>Sandheden</b> for og ved Siden af ham , han felv , som det evige Liv , ved og i hvem alle Ting ere , tager med sin Nand Bolig i vore Hjerter , og for hvert Skridt , vi tage paa Veien , for hvert <b>Blik</b> , vi gjsre i og ved hans Lhs , opliver hans evige Guddomskraft os . — De Ord : „Ingen kommer til Faderen , uden ved mig " udelukke alle dem fra det levende Samfund med Gud i <b>Kjcerlighed</b> og Kundskab , der uden Ehristus og udenfor ham strcebe derhen ; ikke faameget Saadanne , der al » drig have havt Anledning til at Icere ham at kjende ( thi den forberedende Aabenbarelse, de have erholdt , er jo udgaaen fra Ehristo , som det evige Ord ( C . 1 , 4 , f1g , ) ; de vare og ere faaledes i Fcerd med at gaae til Faderen ) , men meget mere Saadanne, der have tillukket sit <b>Hjerte</b> for hans Forlosning . Dog ogsaa om de Fsrste staaer det ifslge dette Bud fast , at der i , eller , om det er muligt , efter dette Liv ikke staaer dem noget Samfund med Gud aabent udenfor Christus . 2. V . 7 , De begyndte nu at kjende ham , eftersom de fra nu af begyndte at fane nit klarere Indblik i Frelserens eget Vasen . Sml . C . 8 , 19. 3. V , 8. Dette Philippi Forlangende viser , at Disciplene endnu stedse tcenkte sig Faderen ved Siden af , ikke i ham . Hos lesum eller formedelst ham haabede de at lcere Faderen at kjende , maaflee endog paa udvortes Mande , i en lysende Aabenbarelse . 4. V . 9. Sml . C . 1. 1. Indl . „ Havde Christus ikke vceret af samme Vcesen , fom Faderen , havde hlln ikke kunnet sige dette . For at bruye et Ezempel til Oplysning : Ingen , der ttke fjender Guldet , tan loere dettes Voeseu at fjende r Sslvet ; thi den ene Natur aabenbarer sig jo ikke i den anden , " CKrysostomus . 5. V . 10. Mellem Faderen og S onn en sinder et levende Vezelforhold Sted Ssnnen behsvede Faderen , eftersom han fra Evighed har faaet sit Liv af Faderen ( E . 5 , 26 , ) ved den evige Fodsel ; Faderen behsvede Ssnnen , eftersom han i Ssnnen bliver sig <b>selv</b> aabenbar og Gjenstand for

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016102148012'target=_>, 1876, Psalmebog for Kirke og Hus</a><h4>2145</h4>2. Bel er du som andre arme Adams Sonner syndefuld , Men Gud stmkler sine Arme Mod dig hjertemild og hnld ; Bender dn dig om igjen , Se , saa er han strax din Ben ; Bcer frimodig , Gnd vil tage Mod din Angers Graad og Klage . 3. Mener du , hau er en Love , Som kun torster efter Blod ? <b>Kjærlighed</b> og Gunst at ove , Dertil staar haus Sind og Mod ; Gud han har et Fadersind , Al vor Jammer treenger ind I hans sode Faderhjerte , Gud han foler <b>selv</b> din Smerte . 4. Hm hans Ord : saa <b>sandt</b> jeg lever , Bil jeg ei en Synders Død . Dn , som for Guds Brede bæver , Glæd dig , det har ingen Nod ! Naar tun Gud din Anger ser , Alt hans <b>Hjerte</b> mod dig ler , Mildt han <b>moder</b> Syndre bange , Aldrig komme de for mange . 5. Ingen Hyrde saa kan rende Efter det fortabte Faar ; Kunde du Guds <b>Hjerte</b> ljeude , Hvordan det i Lne staar , Naar han ved

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009082403015'target=_>Linderot, Lars, 1850, Lars Linderots Prædikener paa alle Søn- og Helligdage</a><h4>746</h4>sig saa god af Naturen , saa retferdig af sig <b>selv</b> , at han ikke behover nogen Bedring , idetmindste ikke nogen <b>sand</b> Bedring , ved hvilken Hjertet <b>selv</b> forandres . O ! hvilken begredelig Stolthed , ligesaa almindelig blandt Sikkerhedens Bsrn , som den visseligen udelukker fra Himmeriges Salighed? Hsr mig I , som ere stolte i Hjertet , I som ere langt fra Retferdighed ( Es . 46 , 12 ) . Hvorfor hovmoder sig Jord og Aske ? ( Syr . 10 , 9 ) . Er ikke Jorden Eders <b>Moder</b> , Synden Eders Hovedegenstab , Fortabelse Eders Arv , Helvede Eders fortjente Lon ! Hvad have I af Eder <b>selv</b> andet end Synd og Fordervelse ? Have I ved egen Kraft Fortrin i andre Ting end i saadant , som fortsrner Gud og foraarsager Eders Fordervelse ? Og have I noget Godt , saa have I faaet det af Gud . Kunne I udrette noget Godt , saa er det ved Guds Haand , der bruger Eder som et Middel til sine vise Hensigters Fremme . Hvi rose I Eder da ? Hvordan kunne I vove at paaberaabe Eder nogen egen Fortjeneste ' ? O ! vider , at Gud staaer de Hofferdige imod , og haster med at ydmyge Eder under sin veldige Haand ( 1 Petr . 5 , 5. 6 ) . Ja , Guds naadige Aand stal nedstaae den egenkjerlige Synders stolte Indbildninger og nedkue hans opbleste Sind . Folelsen af Syndens Elendighed ' stal vekkes i Hjertet , og bliver den ret inderlig og levende , saa stiger Synderen gjerne ned fra sin indbildte Hoide , og cmseer sig for en af de mest Nedrige . De <b>Oine</b> , der for vare Mukkede for hans egen Brost og Skrobelighed , blive nu aabnede , saa at han faaer se en Fordervelse , som er übeskrivelig , og en Syndeskyld , der allerede i Begyndelsen af Regningen belober sig til ti tusind Pund . Nu sinder han , at han er syndefuld og fortjener at vorde fordsmt . Nu erkjender han med Paulus : leg er den groveste iblandt Syndere ( 1 Timoth . 1 , 15 ) . Nu raaber han af sit <b>Hjertes</b> inderste Dyb om Naade og Forbarmelse . Og se , nu bliver Jesus ved sin Forsoningens Naade dyrebar og efterlengtet. I denne Sjelsnod kan ingen Anden hjelpe end han . Ud af denne Elendighed kan ingen Anden redde end han . Den Naade , som han tiltrenger , kan ingen Anden meddele end han . Det er da til ham , at det bekymrede <b>Hjerte</b> , beveget af hans naadefulde Forjettelser , tager sin Tilstugt . Det er for hans Naadetrone , at den dybt nedbsiede

<h4>3066</h4>hvor « verdigen de behandlede ham , netop fordi han elstede dem , og hvor de gjengjeldte hans <b>Kjerlighed</b> og Godhed med Had og Ondstab . Han taler om Frelserens Sjelekvaler og haarde Arbeide , som om han <b>selv</b> havde veret hos ham Natten til Langfredag i Gethsemane , og seet hvor han der vankede om i de storste Kvaler , og tilsidst kastede sig ned paa Jorden for at bede , svedende blodig Sved . Han taler om Frelserens stille Taushed under de svereste og meest lognagtige Beskyldninger , om hans Udfsrelse til Golgatha , som om han <b>selv</b> havde seet , med hvilken Taalmodighed det Guds Lam bar de storste Fornermelser , man nogensinde kunde tilfole ham , ja , som om han <b>selv</b> havde ledsaget de gredende <b>Kvinder</b> fra Jerusalems Port til Golgathas Bjerg , og seet sin Saligheds Gud , uden at ytre det ringeste forbandende Onste over hans Plagere , at bere det tunge Kors paa de smertende Skuldre . Han taler om Frelserens skammelige Ophengen mellem tvende Misdedere , som om han felv med Johannes og Maria havde staaet ved Korsets Fod , og seet den Allerhelligste at blive henregnet til Misdederes Tal . Han taler om Frelserens Forbsn for sine frekke Mordere , som om han <b>selv</b> havde hort disse naadefulde Ord fra hans blege Mund : Fader ! forlad dem , thi de vide ikke hvad de gjore . Han taler om Frelserens Begravelse , som om han <b>selv</b> havde hjulpet Josef og Nikodemus med at legge ham i den nye Grav , og i vore Indgcmgsord taler han om Jesu Saar , som om han med sine egne <b>Oine</b> havde seet ham i den ynkverdige Stilling , da Pilatus lod ham lede hen ud til loderne , og sagde : see hvilket Menneske! Derfor siger han : vi have faaet Legedom ved hans Saar . Netop saaledes som dette var for Esaias ' s arme <b>Hjerte</b> , netop saaledes maa det ogsaa vere for dit <b>Hjerte</b> og Sind , min Ven , ihvo du er , som horer mig , hvis det stal staae vel til Med dig : den sande Frelser maa staae for dig , som om du havde seet ham netop paa den Dag , da han led for dine og al Verdens Synder . Han fik vel sine Saar for flere Hundrede Aar siden , men hvis han ikke stal have faaet dem forgjeves , hvis du stal have noget Gavn af dem for din arme Sjel , saa maa det ikke vere dig anderledes , end som om han havde faaet dem idag : ikke anderledes , end som om han idag var bleven hudflettet , idag tornekronet ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011062208060'target=_>Tholuck, August, 1847, Kristelige Andagtstimer</a><h4>563</h4>fsrste Tid a f ! " O Herre , hvor tidt og mange Gange har du ikke forandret Skikkclse for min Skyld , bestandig er du kommen mig imsde paa en anden Vej , at dog mit Vje maatte kjende dig og mit <b>Hjerte</b> fornemme dig ! Mangengang gik det mig forn Disciftlene i Emaus ( Luk . 24 , 13 ftg . ) ; mit <b>Hjerte</b> brcrudte , og dog vidste jeg ikke , at det var dig . Nn kjendcr jeg dig i alle dine Forklcedninger ; nu feer jeg dig ftaa alle Vcje . Ia det er en Velsignelse at vcere opvoret indenfor den kristclige Kirkes Grcendser , en Velsignelse <b>selv</b> i vore Dage , da Zions Mure cre saa forfalvne . Naar jeg nn saaledes tcenker mig tilbage , hvor mange Indtryk af Kristi Aand cre ikke da nvittcrligen og uvilkaarligcn blevne mig til Deel ! Jeg kan dog i Virkeligheden sige , at dens Modcrbryst bartad ligesaa tidligt begyndte at erncere mig aaudeligcn , som min kjodeligc Moders Bryst begyndte at erncere mig legemligen . De Bonner , som min <b>Moder</b> lcrrte mig , mange fromme Mennesiers Ercmpel , som jeg Icertc at kjende af Herrens Kirkes Historie , Konsirmationsunderviisningen, saa mangen Gudstjeneste , den kristelige Skik og Brug — alt dette har dog indvirket ftaa mig . Og dengang saa du , evige <b>Kjcerlighed</b> , som ideligen har draget mig til dig , virkelig drog mig til dit <b>Hjerte</b> , og jeg sinede ind i dit aabne Aasyn , da samlede alle disse enkelte Kjcerlighedsstraaler sig tilhobe , og det Lys , som faldt paa min Datid , gjorde med det samme min Fortid og min Frcmtid lys for mig . — Vilde jeg slutte ester min egen Erfaring , saa skulde jeg vcere tilbojelig til at antage , at der i ethvert Menneskes Liv er siere Spor af Guds Naade for dets Omvendelse , end det <b>selv</b> troer og aner . Det gaacr dermed som med Lyset , der indvirker oplivcnde paa Mcnnestct , uden at dette mcrrkcr , hvorfra Oplivelsen kommer . Ter er , som om man saae og greb Frelseren gjcunem et Clor , og Omvendelsen bevirkcr kun , ac Slorcc bliver taget bort . Man kan jo tvivle om en <b>Sandhed</b> , som man dog ikke kan ocuegte : man kan , som Apostelen siger , vcere greben af Kristus uden dog at gribe barn . Der er en forberedende Naade , der indvirker paa Mennesket og senter sig fast i det , udeu at det vecd deraf . Saaledes har du , min Gud , dengang jeg endnu ikke kjendte dig , ~ draget mig til dig af lut te r Miskundhed . "

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012073108081'target=_>Hauge, Hans Nielsen, 1867, Christelige Lære</a><h4>175</h4>gede Christus sig saa meget mere , at enhver stulde tåge Gxempel, at forringe sig og foragte Verdens Herlighed ; Me tragte ester det Hoie , men holde sig til det Ringe ; hvorom kunde vcere meget at tale ; . men enhver som vil , kan fatte dette at betcenke og betragte vor Evangelium paa denne Maade ; er eller bliver overbevist af det Anfsrte ; og da det ikle vilde hjcelpe , om vi forringede os aldrig saa meget udvortes ; som mange har såldet paa saudan en Elendighed , og stolet paa den udvortes Ringhed , afsondret sig fra Mennestene , eller fort et Skin af udvortes Foragt paa Verden , og derved virre aandelige Hovmodige ; haver syntes de da stulde vcere store Guds Born , dog har gjort det af et Skin at vise for Verden , har tåget iagt den udvortes Renselse paa Legemet , og forglemt det indvortes , hvilket stulde dog fsrst foregaa . Lue . 11 Cap . 39 V . saa fulgte det andet siden . Vi vil derfor betragte den indvortes Renselse , ny Fodsel , i Aand og <b>Sandhed</b> , at saa renset baade det Indvortes og Udvortes , at blive vor Frelser lig , saa han kunde ligge i vor Krybbe eller <b>Hjerte</b> , og der saa Herberge ; at alle vore Synds Lyster , som for haver vceret til Hvile , for de urene Aander og herberget dem , maatte nu blive stikket til at herbergere Guds Sons <b>Moder</b> , at Jesus med sin guddommellge Kraft kunde uddrive Fienden , dampe det Stråa eller Kjsvcts Lyster hvor Satan har havi sin Fsde i ; da vi kunde vev den Helligaands Undfangelse blive Jesu hellige <b>Moder</b> , saa han med Faderen i vore Sjele , kunde formedelst Troen paa Ordet bo i os , at vi holdte hans Ord og blev i hans <b>Kjcerlighed</b> ; Joh . 14 Cap . 23 V . at han som havde den guddommelige Herlighed , vilde saaledes forringe sig <b>selv</b> , for at herliggjore os . Dette Store bestuttede Guds Raad , som Herren han havde af Gvighed befluttet , for Vertens Grundvold blev lagt ; Eph . 1 Cap . 14 V . , thi da han stabte vore forste Foraldre i sit Billede af Jord , og gav dem en Sjel ; denne forste Adam var da af Jorden jordisi , 1 Cor . 15 Cap . 44 V . at endfljont Gud stabte dem i sit Billede med fri Villie til at gjore hvad han vilde , og gav ham Magt over alle stabte Ting , med Forstand til at regjere det der var paa Jorden ; saa var det dog ingen guddommelig Natur , forenet med guddommelig Ggcnflaber af guddommelig , himmelsk Kjcerlighed, at Gud elstede dem som sine Born , og de ham som sin Fader ; thi de vare Me af guddommelig Art , men menneflelige,

<h4>1025</h4>/ " / " " bet er Falflhed og Hykleri , da de alene g saadant som de bedrager sig felv med , og vil nore , det er : af Djevelens Indbildning , de stoler Mne Taarer eller Glader , som de dog <b>selv</b> haver „ da det ikle flyder fra Hjertet , som kjendes paa deres Uftadighed og Gjenkjcerlighed , hvor liden den er , ja intet , men tvertimod har de Had , Vrede og Fiendstab til Christi Aand , Tjenere og Befalninger ; thi nåar vi taler om Christi Gjerning at gjere , saa lader Frugten sig tilsyne , at de bliver tungsindige og uvillige , ja vred , for at en fordrer Troens Frugter as dem ; derfor er deres Tro dsd , da Gjerningen og Christi Dyder er borte , de er nogle ustadig ? Ror , som de blode kjedelige Born , der glader sig nåar de faar eller horer , at de stal saa noget godt , men er uvillige nåar de fordres at gjsre ; de er beangstet nåar de ser nogen lide , og bange af Kjodets Kjelenhcd at lide noget <b>selv</b> - de er as TjenejwKvindens Sen , og det Jerusalem hernedentil , som er i Trceldom , hvilket vi laste om i Midfaste Ssndags Epistel ; det maa derfor hede til dem : I Dsttre af Jerusalem , grceder ilke over mig , men grader over Vder <b>selv</b> , Eders eget onde <b>Hjerte</b> og Eders Born , som I avler i Syndens Elendighed og opdrager i samme Ondflab ; det flulde i bitterlig begrade med <b>sand</b> Poenitense og Omvendelse ; thi den Tid stal komme , at de stal sige : Salig er dct Liv som ikte fsdte de onde Lyfter i fit <b>Hjerte</b> , og det Bryst som ikte af Syndens <b>Kjarlighed</b> gav <b>Die</b> til andre Verdens Bsrns Bestyrkelse i det Onde ; thi de som det har gjort , stal Guds Vrede komme over ; saa de ffal nedstyrtes i Afgrunden , og Bjergene falde paa dem , da Synden stal blive hoi opfireven , saa den stal fljule dem fra al Trost og Glade ; at de ikte mere Glade stal faa , men de tunge Bjergers Byrde trykker og klammer dem med Evigheds Piner ; thi nåar det gaar saa med det gronne Tra , at den Hsiestes Son , som var fri fra al Synd , han udperfes og pines saa haardt , hvad vil det da blive med det torre , som er fyldt med al Ondstad ? mon de kunde tanke at undgaa? Nei langt fra . Lad os derfor ikle vare med dem , som vil grade over Christi Lidelse , enten for at de saa andre pinede ham , eller betcenker at de <b>selv</b> havde kommet ham til at lide Plager for deres Ovenradelsers Skyld , Es . 58 C . BV . dog ikte vil fornagte sig <b>selv</b> , elfle Gud og sin Ncrstc , bade

<h4>2326</h4>og i Jerusalem , nemlig : det himmelske , der er det Raad ar undkomme , loels 2 Cap . 15 V , og de stal bo paa Zions- Bjerg , som vandrer fuldkommeligen og taler <b>Sandhed</b> i sit <b>Hjerte</b> , ikle tåger sin Sjel forfengelig i sin Mund . 24 Ps . Det er Tid at fly did iblandt Guds Born , der bygger paa Klippen , eller Bjerget Christus , og ere haarde , at de Onde ikte stal forstrcekke os , eller boie as Veicn , Ps . 44 , og hvo som er paa Tåget han stiger ilke ned , at hente noget af sit Hus ; thi nåar Eders Sind er oploftet fra Jorden og er kommen i den hellige Kirkes Samfund , som Guds Kraft haver dedellet , da stig ikte ned i det kjodclige Hns , eller Synds Lyster i Verden, at tåge noget derfrå , elste og udvcelge ; tbi det cr urent og scenker Sjclen ned i Fordcervelsen igjen . Hvo som er paa Ågeren vender ikle tilbage at hente sine Klcedcr . Thi nåar vi cr med Loth ndvandret fra Sodomas Ondstab , saa vi kunde undgaa Ilden , og itte blive fordcervet ; 1 Mose B . 19 C . da maa vi erindre Jesu Advarsel : lom Loths <b>Hustru</b> ihu : Lue . 17 C . 32 V . saa vi ikte ser tilbage , at vilde have uoget med af vore besmittede Klceder , eller denne Verdens Gode , hvis <b>Kjcerlighed</b> vi haver klcedt vor Sjel med for . Men ve de Frugtsommelige , og de som give <b>Die</b> ! i de Dage ; thi nåar Doelceggelsen kommer , ve ! da dem , som ere frngtsommelige af Synden og haver givet de onde Begjcerligbeder Rnm , der ver undfanget og fsdt Synd , givet dem <b>Die</b> af ' sit Hjertens Bryst ; det fodcr Dsden ; men beder at Eders Flugt stal ikte ste om Vinteren ; thi nåar I er forkjolet i Eders Aand , saa Guds Kjoerligheds Varme var borte formedelst Synden , hvor ilde vilde da Eders Flugt blive , hvor skulde I fly hen ? nåar I ikle elstede , og var eistet af Gud ; derfor maa I bede med David : Herre ! tag mig Me bort midt i mine Dage . 102 Ps . 25 V . Ei heller om Sabbathen , da I skulde bvile . dog maatte Arbeide ; thi da stal vorde saa stor Trcengsel , , fom ikte haver vceret for Verdens Begyndelse hidindtil , som ikte heller stal blive . Dette kan og siges nu : at Trengselen haver vceret saa stor over Gnds Born . af de falste Lccrere og andre Forforere, med fmigrende Ord , Hykleri og Skriftens Fordreielfe , saa de Trofaste vare sdelagtc , saa der var neppe et levende Gnds Barn at finde , for Gud nu atter befsgte fit Folk , med sin Kraft , bevcegede alle Hedningerne at de med Lcengsel kan lomme til Herrens Hus og nyde hans Gode . Hag . 2 C . 7

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010111006094'target=_>Arndt, Friedrich, 1876, Kristelige Betragtninger til hver Dag i Aaret</a><h4>532</h4>I <b>Sandhed</b> , <b>Kjcerlighed</b> , Intet uden <b>Kjcerlighed</b> var Grundtonen i Jesu Kristi Liv fra Begynnelsen til Enden . Denne <b>Kjcerlighed</b> viste sig fremfor Alt som en m ed-lidend e <b>Kjcerlighed</b> . At han forlod Himmelen og steg ned til vor Elendighet ) , at han blev fM i Ringhed , vandrede i Ringhed paa Jorden , at han tog imod Gaver af de fromme <b>Kvinder</b> , som fulgte ham , at han delte de Svagheder og Smerter , som ere forenede med vort jordiske Liv , at han bar Dagens Byrde under Folkets Larm og ofrede sin Nattero , naar lcerelystne Sjcele vilde hsre ham ; at den hvasse Tornekrone er trykket paa hans Hoved , at hans livsalige Ansigt er bleven slaaet med Rpret , at hans Legeme under Svsben er bleven bedcekket med Blod og hans Hcender og Fadder gjennemborede ; at han endelig vansmcegtende paa Korset befalede sin Aand i Faderens Hcender : i alt dette har han lidt vort Livs Nsd med os . Men mere endnu end den legemlige Nod , har han delt og fM vor Sjcelenpd med os . Ak , hvert Bjeblik , hvori hans rene og hellige Sjcel dvcelede i denne Verden fuld af Frafald og Opwr mod Gud , maatte jo etSvcerd gjennemtrcenge den ! Hine ensomme Timer , som han har tilbragt i Stilhed med sin Fader , hvor maa de have vceret opfyldte af den smertelige Melse for en Verden , forn er sin Undergang ncer ! Og nu i hans daglige Omgang med den forvendte Slcegt , snart ligeover for hykkelske Skriftkloges raa Hovmod og bitre Haan , snart i Dissiplenes Kreds ligeoverfor de stewe , dorske , jordiske Hjcerter, intetsteds en Sjcel , som ret forstaar ham , som kan fatte Storheden af hans Smerte . Endelig i hans sidste Timer , hvor udgyder sig da det kjcerlighedsrige <b>Hjcerte</b> i Kredsen af dem , som han kaldte sine egne ; hvor ligger Kjcerlighedm under Fodtvcetningen paa Knee <b>selv</b> for en Judas ; hvor maa han opleve det , at hans fortroligste Venner sove ind i Getsemane, endogsaa Johannes , medens han sveder den blodige

<h4>893</h4>Paaskedags Wen var brudt frem . Maria Magdelena havde sst Herren , de ovrige <b>Kvinder</b> havde modtaget hans Hilsen , Peder havde haft en Aabenbarelse af ham . Kleofas og hans Ledsager havde i flere Timer nydt hans Omgang om Eftermiddagen . Nu ere de alle forsamlede moen lukte Dore , foroybede i Samtaler om den store Dags Begivenheder , den Ene henrykt , den Anden beklemt , denne fortcellende , hin svsrgende, her troende , der tvivlende . Da kommer Herren <b>selv</b> og staar vluoselig midt iblandt dem og siger : Fred vcere med Eder ! Ak , efter saa mange alvorlige Ovtrin , ester den store Bevcegelse , hvori de havde vcrret , trcengte de visselig yl Fred . Og Han bragte dem Freden . Hans Ord er stedfe tillige Handling . O himmelske Vtildhed ! Saaledes gaar Herren altid efter de Bekymrede og Anfoegtede . Saaledes kjender han deres Noo , deres Smerte , og giver dem , hvad de fattes . Tom h w no et kom han aldrig , og det Bedste , han kan bringe os , det som gjsr Aftenen til Helligaften og Hviledagen til Hviledag , er netop hans Fred . Denne Fred er den bedste Paaskegave ;  den overgaar al Forstand og bevarer <b>Hjcerter</b> og Tanker i Kristus Jesus . " — Efterat Jesus havde hilset Avostlerne :  Fred vcere med Eder ! " soger han at tilegne dem denne Hilsen og at gjsre den levende og frugtbar i deres <b>Hjcerter</b> ; han forjager deres Frygt ; han foreholder dem i al <b>Kjcerlighed</b> deres overtroiske Tanker , da de mente at se en Aand eller et Gjenfcerd ; han fremforsr det ene Bevis efter det andet , for at befceste Troen paa hans Opstandelfe i deres <b>Hjcerter</b> ; han meddeler dem en klar Indsigt i Guds Naades Plan og Husholdning og Hans evige Raad og Villie til vor Forløsning , saa at deres Erkjendelse mere og mere udvides og begrundes ; han viser dem , at kun den Erkjendelse er <b>sand</b> og

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016103148541'target=_>Berg, Ole, 1863, Prædikener over Evangelietexterne paa Kirkeaarets Søn- og Helligdage</a><h4>999</h4>Morgendrsin fornam Guds Narhed , stjondt du vidste det ei ; da dit <b>Oie</b> først hcevede sig opad mod Himmelen med den Tanke : der boer Gud , — da var der allerede saaet i dit <b>Hjerte</b> af den nforkrcent ' clige Sad , som , saasandt den ikke voldelige« dråbes , eller de deraf sig ndviklende Spirer kvcrles , bliver til et evigt Liv . Da du af eu kjcrrlig <b>Moder</b> eller Fader eller Ven og Beskytter Icerte at stamme : „ Fader vor , du , som er i Himlene , helliget vorde dit Navn , " saa blev der saaet i dit <b>Hjertes</b> Ager , og siden — at ! hvor mange Gauge er der ikke saaet . Nu hvorledes , min tjare Tilhører ? Har Sa ' den spiret vakkert ? staaer den frist og saftig ? Lover Ageren godt Udbytte til Hosten ? Er der Tegn til Frngter ? Frugterne veed du jo uok komme strar tilsyne paa Aandens Agerland , og Aandens Frugt er jo , som du veed , <b>Kjærlighed</b> , Glade , Fred , Laugmodighed, Mildhed , Godhed , Tro , Sagtmodighed , Afholdenhed . Eller hvorledes er det med dig ? Er det maastce visnet , hvad der blev saaet i dit <b>Hjerte</b> ? O , bliver dn da ikte <b>selv</b> bedrovet over dig <b>selv</b> ved at scc , at de gronne , lovende Spirer ere dode og borte ? Har du ikte Medlidenbed med dig <b>selv</b> ? Gud har i ethvert Fald Medlidenhed med dig — han saaer idag paany . Det kommer an paa , hvorledes dit <b>Hjertes</b> lordbuud er uu — uu i deune Stund . Ligner det en Vei ? Du veed jo uok , at paa Veicn vaudrer baade Folk og Dyr , og Himmelens Fugle have let for at uappc bort deu Sced , som er udstroct der , og som ligger nbedcrtket af Jorden . Ligner dit <b>Hjerte</b> en Vei i denne Stund ? Vandrer der fremmede Tanker om derinde, og kan dn marke , om Djævelcn er paafcerde for at nappe Ordet fra dig , at det ikke for dig stal blive et Livsens Ord til Liv ? Ligner dit <b>Hjerte</b> ikke en Vei , saa se til , at det ikte kommer til at ligne hun Klippegrund , hvor Jordlaget kun var tyndt . Det kan dn bedst komme efter , naar dn gaacr herfra , og naar dn er kommen i dit Hjem . Oieblikket efter Ordets Horelse er det farlige , mangen Gang det afgjorende Oicblik . Eller hvorledes er det med dig , min Tilhører ! ere Tornene allerede vorcde op derinde saa tykke og tcette og store og mægtigc, at det er at befrygte , at Ordet , som er saaet idag , vil kvceles , og den hellige Sced spildes ? Det gaaer an at rykke disse Torne op ; det er et moisommeligt , maastee ogsaa et langvarigt Arbeide , men det gaaer an . Tag Arbeidet for strar , og bliver du trcet og mat , saa kom Herreus Ord til Paulus ihu : „ Min Naade er dig not , tbi min Kraft fuldkommes i Skrobelighed . " Ved Guds Naade kau du frelses ; ja , endnu kau du frelses . Og du , min kjære Tilhører ! som Gud har forundt den Glade at sce , at det i dit <b>Hjerte</b> allerede begynder

<h4>1310</h4>og med Hjertets fulde Varme , som de jo maatte gjore , hvis Sagen stod for dcm i sin hele Alvorlighed . Deraf den saa almindelige Lunkenhed i den christelige Tro overhovedet og i det christelige Haab og den christelige <b>Kjærlighed</b> ; deraf den megm Halten til begge Sider ; man vil ikke ganske bryde med Gno og Evigheden , men heller ikke med Verden og Tiden . Ja , jeg ' tor vel sige , at hos os Alle er der Tider , da him saa vigtige og alvorsfulde Gjenstand for Troen ligesom trader i Baggrunden , medens denne Tids verlendc Spil fangster vort <b>Blik</b> og fortrinsviis optager Hjertet . Vi kunne derfor Alle trcenge saa saare til at paamindes og opvarmes ved de store Minder , som denne . Fest sigter til at fornye , og Alle trcenge vi til at vore og befæstes i Troen paa Kjødets Opstandelse og det evige Liv . Men hvorledes stal denne Tro da vel kunne stadfccstes og befæstes ? Visselig er det til den Ende gavnligt tidt og ofte at foroge i sin Sjæl Mindet om Jesu Opstandelse fra de Dode og at lægge sig paa Hjertet , at ingen Begivenhed er stærkere beviist og stadscestet end denne . Men ligeoverfor Vantroen , som altid vil trcenge sig frem , ere saadcmne Overvejelser stundom afmccgtige ; det viser sig at vare bogstavelig <b>sandt</b> , hvad Jesus lader Abraham sige til den rige Mand : „ Dersom de ilke troe Moses og Propheternc , saa ville de hetter ikte troe , om en Dod kom tilbage fra de Dodcs Nige . " Saa . stærk er Tvivlens og Vantroens Magt . Heller ikte ville de Slntningsrcekker , som for Enhver , som troer paa en levende Gud , saa uaturlig tilbyde sig , og som synes at gjore det saa klart , at Mennesket maa være bestemt til en anden og høiere Leven end denne jordiske og timelige , saasandt der er en Gno til , og han — som naturligen folger af selve Gudsbegrebet — er alviis , god og retfcerdig , vare tilstrekkelige til at holde Stand imod den tvivlende Forstand og det vantroe <b>Hjertes</b> forellede Angreb . Men hvorledes skulle vi da vinde frem til Fasthed i vor Tro og i vort Haab ? Saaledes , mener jeg , at vi folge det Raad af Frelseren først at give Agt paa , hvad Moses og Prophetcrne have at sige os , give Agt paa Gnds Villie saaledes , som denne umiskjendeligcn og nnægtcligcn er os aabcubarct i vor Samvittighed og i Guds hellige Lov og i Naturlivets og det « åbenbarede Ords dunkle Profthctier , eller med andre Ord : saaledes , at vi forsoge at adlyde det af Guds Ord , som vi nodes til at erkjende og boie os under , og give dets Virkning Rum i Hjertet . Christus har <b>selv</b> anbefalet os denne Vei , som en Vei , der sikkert leder os til ham ; „ Troe I Moses , saa maae I og troe paa mig ; thi han har skrevet om mig , " siger han etsteds , og paa et andet Sted siger han :

<h4>1798</h4>Vi have nu seet , hvor Meget der er at see hen til for at kunne fcelde en retfcerdig og upartisk Dom over Andre . Men jeg maa endnu minde dig , som foler Lyst til at domme , om dette , at dn , saasaudt du vil domme eh risteligt , naar du fordommer, altid maa gjore det med <b>Kjærlighedens</b> Sorg , med inderlig Bedrovelse . I denne Henseende vil jeg erindre dig om , at det jo er en Broder , en Sostcr i Herren du dommer , at det er en ved din Frelsers Naade gjenlost Sjal , som du troer at maatte fordomme . Og dette kan sandeligen ikke vare nogen Glade for nogen Christen . Den , som har faaet sin egen Sjal frelst ved Guds Barmhjertighed og ene og alene ved Guds Barmhjertighed , maa fole sig i <b>Kjarlighed</b> knyttet til de andre Sjale , som ogsaa have Barmhjertighed behov , og maa inderligen onsic , at der ogsaa maa vederfares dem Barmhjertighed. Men med et saadant Sind , med saadanne Tanker og Folelser , kan man kun med Sorg og dyb Medlidenhed skride til at maatte domme . Og , naar du dommer din Broder eller Soster , saa lad den menneskelige Svaghed , som du maa domme , tjene til at aabne <b>Oiet</b> for din egen Skrobclighed . Du vil da sikkert finde nok at domme hos dig <b>selv</b> , og Dommen , som du maa fcelde over dig <b>selv</b> , vil gjore din Dom over Næsten mildere og kjærligerc og fylde dit <b>Hjerte</b> med Sorg over din egen Syndighed og med Sorg over , at du maa domme Synden hos en Anden . Og endeligen — kom ihu , at du aldrig dommer , uden fordi <b>Sandheden</b> krcever det af dig som et Offer . Naar du saa dommer , fordi du maa domme , saa tor du haabe , at du maasiee ogsaa ved din Dommen kan bidrage til at frelse ; og det Maal bor du altid have for <b>Die</b> ved al din Dommen . Din Dom maa derfor aldrig lyde anderledes , end du vilde kunne ytre dig i den formecntlige Skyldiges egen Nærværelse , og glem aldrig , at Gud altid er tilstede som et levende Vidne og som alle usande og ukjccrlige Dommes Hevner . Du vil maasiee sige , min Medchristcn ! at alle disse Negler forudsættc Fuldkommenhed hos den Dommende . Ja , det gjore de ogsaa ; det er sandeligen ikke uden Grund , at Apostelen Jacob siger ( lac . 3 , 2 ) : „ dersom Nogen ikke stoder an i sin Tale , denne er en fuldkommen Mand , istand til at holde det ganske Legeme i Tomme . " Men det er jo ogsaa Fordringen , som Christus opstiller til Sine : „ varer fuldkomne , som eders Fader i Himmelen er fuldkommen ! " Og du feiler hoiligen , naar du , fordi Fuldkommenheden ikke kan naaes her i Stovets Land , troer dig berettiget til for din Bekvemmeligheds Skyld at slaae Noget af paa Fordringen , eller indbilder dig at have gjort den 20

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016031548122'target=_>Landstad, M.B., 1873, Kirkesalmebog efter offentlig Foranstaltning</a><h4>282</h4>ham med Lomgsel <b>sand</b> , Hans <b>Kjærlighed</b> at voelge ud , I Aanden brcendende for Gud ! 12. Elsk ham , som eder elstte saa , Gjor ham igjen hvad I formåa , Med gode Lofter fra ham gak , Velsigner ham og siger Tak . 13. O himmelske Miskundheds Mand , Du Livsens Haab i Dodens Land , Du Naade-Kilde fuld af Trost , Du sode Gloede i mit Bryst ! 14. Jeg taler med uværdig Mund , Men tie kan jeg ingenlund , Min <b>Kjærlighed</b> den tvinger mig , For Hjertet er saa fuldt af dig . 15. Med dem , dig smage , saa det sier , De længes altid efter meer , Du aldrig dem af Minde gaar , Til dig al deres Attråa staar . 16. Jesu , Guds Engles sode Kvad , I Oret Sang , som gjor En glad , I Munden Honning underlig , I Hjertet Dugg af Himmerig ! 17. Min Hu staar altio efter dig , O Jesu min , kom vist til mig l Nåar vil du gjore mig saa glad ? Moet mig med dig , det er min Mad ! 18. Her gaar jeg syg af stor Attråa , Kan jeg et venligt <b>Oie</b> faa , Det drypper Honning i mit Saar , Jeg lives op , og styrket staar . 19. Jesu , den Hjertens Skat , jeg veed , Jesu , du himmelsk Mdighed , Og Godhed übegribelig , Din <b>Kjærlighed</b> har tvunget mig ! 20. Dig , Jesu , har jeg valt mig ud , Jeg være vil din Hjertens Brud , Bort fra mig <b>selv</b> jeg gjerne dor , Og lever dig , som det sig bor . 21. O Jesu , op dit <b>Hjerte</b> luk , Og lyt til mine Lcengsels Suk , Saa inderlig med Graad paa Kind Jeg raaber : Jesu , Jesu min ! 22. Ja hvor jeg om i Verden gaar , Min Hu ril lesum ene staar ; Hvor blev jeg glad , der jeg ham fandt , Som Dugg for Solen Sorgen svandt !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009021812004'target=_>Lous, M.C. Theodor, 1860, Haandbog i den christelige Børnelærdom for Confirmander og Confirmerede</a><h4>669</h4>Den anden Vildfarelse , jeg her vilde advare mod , er , at man Me maa trettes <b>selv</b> da , nåar vore edleste Bestrebelser , vore kjerligste Forsetter miskjendes , ja , om vi end tennes med Had og Foragt for <b>Kjerlighed</b> ; men , med Frelferens Exempel for <b>Oie</b> , langmodig og taalmodig virte i <b>sand</b> Broderkjerlighed for Saamange , vi formcme, felv om vi Me stue Frugter af vor Virken , om vi Me hoste anden Lsn , end vort eget <b>Hjertes</b> glade Bevidsthed om at have handlet ret , efter Skriftens Ord i 2 Thess . 3 , 13. : „ I Brsdre ! bliver Me trette af at gjsre Godt " . Necmder siger derom : „ Alle overspendte Fordringer til Andre , eller til Verden overhovedet , have sin Grund deri , at Mennestet er sig <b>selv</b> fremmed , Me erkjender, hvormegen Grund han endnu har til at sukke over sig <b>selv</b> , hvormeget at forbedre hos sig <b>selv</b> , og deri , at han vil have og nyde Himmelen allerede her paa Jorden, og Me ede sit Brsd i sit Ansigts Sved . " 3 ) Fremdeles maa vor <b>Kjerlighed</b> til Nesten vere almindelig , o : den maa gjelde Alle , saa vi , faavidt vi formaae , virke til Alles Vel , og dette uden Persons Anseelse , altsaa uden at see hen til Religion eller Landsmandstab , Bekjendtstab eller Ubekjendtstab , kort , uden utlade nogensomhelst ydre eller tilfeldig Omstcrndighed indvirke paa vor <b>Kjerlighed</b> ; lac . 2 , 9. : „ Men dersom I ansee Personer , gjsre I Synd , og overbevises af Loven forn Overtredere". Om den Fremmede heder det i 2 Msbg . 10 , 19. og i 3 Msbg . 19 , 34. : „ Den Fremmede , som er fremmed hos Eder , stal vere Eder forn en Indfodt , og du stal elste ham som dig <b>selv</b> ; thi og I have veret Fremmede

<h4>1167</h4>sin Broder , er en Manddraber , og hvad hjelper det os vel , at Menneskene ikke kjende os som Mordere , nåar Herren i Himmelen demmer os som saadanne ? Derfor , min Broder ! vil du prsve dig <b>selv</b> efter dette Bud , da see Me blot til hvad Haanden har gjort , men vend dit <b>Blik</b> indad i Hjertets inderste Grund og sporg dig <b>selv</b> , som for Guds Aafyn : hvad er det , som har boet og boer derinde ? Har aldrig Hadets og Hevngjerrighedens vilde Flamme drevet Broderkjerligheden ud ? Har aldrig Vredens Kulde stukket <b>Kjerlighedens</b> Ild ? Har aldrig Bitterhet » , Hissighet » , Stjeldsord saaret og bedrsvet din Nestes <b>Hjerte</b> ? Og hvo blandt os Alle maa Me da beskjemmet vende <b>Blikket</b> mod Jorden og bekjende : Ak , Herre ! der fattes mig saameget ogsaa i dette Stykke i at kunne bestaae for dig ! Ja , i <b>Sandhed</b> mine Venner , Adams Sed boer i os Alle , og vi trenge Alle til at bede : Herre vor Gud ! tend du felv <b>Kjerlighedens</b> Flamme i vore Hjerter , at den maa drive Had og Fiendstab , Haardhed og Bitterhet » ud af vort Vefen ; lad din <b>Kjerlighed</b> varme os saa , at vi oprigtig lere at elste ham , der lod sit Hjerteblod flyde for vore Synder og felv paa Korfet bad for sine Mordere ; kun da ville vi i <b>Sandhed</b> kunne elste vore Brodre med saadan <b>Kjerlighed</b> , at Had og alle Mordtanker Me kunne faae Rum i vore Hjerter . Dertil hjelpe du os Alle for Jesu Christi Skyld ! Ameu .

<h4>1791</h4>med <b>Kjerlighed</b> ssger at aabne hans <b>Oine</b> for hans Mangler , for derved at rette og forbedre dem . Medens han faaledes blandt Mennestene soger at forsge og befeste hans Wre , gode Navn og Rygte ved at bekjendtgjore og fremheve hans gode Sider der , hvor de Me ere kjendte , at stille dem i et klarere Lys , hvor de kun ufuldkomment kjendes eller paaagtes , er han dog paa den anden Side heller Me blind for hans Feil ; men lader disfe , overalt hvor det kan stee , kun vere en Sag mellem den Feilende og ham <b>selv</b> for fortrolig Meddelelse under fire <b>Oine</b> , paa denne Maade viser han , at han mener Nesten det i <b>Sandhed</b> godt , og at han besjeles af den Broderkjerlighetens Aand , som skildres 1. Cor . 13 , 4 — 7 : „ <b>Kjerligheden</b> er langmodig , er velvillig , <b>Kjerlighed</b> berer itte Nid , <b>Kjerlighed</b> bruger Me Fremfusenhet » , opblefes itte , den er itte usymmelig , syger Me sit Eget , forbittres Me , tenker itte Ondt , den gleder sig itte over Uretferdighed , men gleder sig ved <b>Sandhed</b> , den fordrager Alt , troer Alt , haaber Alt , tanter Alt . " Den Ferdighet » med Lethed at opdage Andres Feil er let erhvervet , men vifer et haardt et ukjerligt <b>Hjerte</b> , Ferdigheten at erkjende og bekjende deres gode Sider er vanskeligere at naae , men er en Frugt af et gjenfsdt fornyet Sind .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015072408025'target=_>Berg, Ole, 1871, Prædikener over Evangelietexterne paa Kirkeaarets Søn- og Helligdage</a><h4>970</h4>MorgendrM fornam Guds Ncerhed , stjMt du vidste det ei- , da dit <b>Oie</b> s > rst hcrvede sig opad mod Himmelen med den Tanke : der boer Gud , — da var der allerede faaet i dit <b>Hjerte</b> af den uforkrænkelige Sced , som , saasandt den ikke voldelige« drcrbes , eller de deraf sig udviklende Spirer kvcrles , bliver til et evigt Liv . Da du af en kjcrrlig <b>Moder</b> eller Fader eller Ven og Beskytter lcerte at stamme : „ Fader vor , du , som er i Himlene , helliget vorde dit Navn , " faa blev der saaet i dit <b>Hjertes</b> Ager , og siden — ak ! hvor mange Gange er der ikke saaet . Nu hvorledes , min kjcere Till Mer ? Har Sceden spiret vakkert ? staacr den frist og saftig ? Lover Ageren godt Udbytte til Hosten ? Er der Tegn til Frugter ? Frugterne veed du jo nok komme strar tilsyne paa Aandens Agerland , og Aandens Frugt er jo , som du veed , <b>Kjcrrlighed</b> , Fred , Langmodighed, Mildhed , Godhed , Tro , Sagtmodighed , Afholdenhed . Eller hvorledes er det med dig ? Er det maastce visnet , hvad der blev saaet i dit <b>Hjerte</b> ? O , bliver du da ikke <b>selv</b> bedrevet over dig <b>selv</b> ved at see , at de grMne , lovende Spirer ere dFde og borte ? Har du ikke Medlidenhed med dig <b>selv</b> ? Gud har i ethvert Fald Medlidenhed med dig — han saaer idag paany . Det kommer an paa , hvorledes dit <b>Hjertes</b> Jordbund er nu — nu i denne Stund . Ligner det en Vei ? Du veed jo nok , at paa Veien vandrer baade Folk og Dyr , og Himmelens Fugle have let for at nappe bort den Sced , som er udstrM der , og som ligger übedcekkct af Jorden . Ligner dit <b>Hjerte</b> en Vei i denne Stund ? Vandrer der fremmede Tanker om derinde, og kan dn mcerke , om Djcevelen er Paafoerde for at nappe Ordet fra dig , at det ikke for dig stal blive et Livsens Ord til Liv ? Ligner dit <b>Hjerte</b> ikke en Vei , saa se til , at det ikke kommer til at ligne hiin Klippegrund , hvor Jordlaget kun var tyndt . Det kan du bedst komme efter , naar du gaaer herfra , og naar dn er kommen i dit Hjem . Aieblikket efter Ordets HMelsc er det farlige , mangen Gang det afgjFrcnoe Meblik . Eller hvorledes er det med dig , min Tilhorer ! ere Tornene allerede voxede op derinde saa tykke og tcette og store og mcegtige, at det er at befrygte , at Ordet , soM er saaet idag , vil kvceles , og den hellige Sced spildes ? Det gaaer an at rykke disse Torne op ; det er et møjsommeligt , maastee ogsaa et langvarigt Arbeide , men det gaaer an . Tag Arbeidet for strar , og bliver du trcet og mat , saa kom Herrens Ord til Paulus ihu : „ Min Naade er dig nok , thi min Kraft fuldkommes i Skrøbelighed . " Ved Guds Naade kan du frelses ; ja , endnu kan du frelses . Og dn , mm kjcere TilhMr ! som Gud har forundt den Glcrde at see , at det i dit <b>Hjerte</b> allerede begynder

<h4>1275</h4>og med Hjertets fulde Varme , som de jo maatte gjFre , hvis Sagen stod for dem i sin hele Alvorlighed . Deraf den faa almindelige Lunkenhed i den christelige Tro overhovedet og i det christelige Haab og den christelige <b>Kjcerlighed</b> ; deraf den megen Halten til begge Sider ; man vil ikke ganske bryde med Gud og Evigheden , men heller Me med Verden og Tiden . Ja , jeg vel sige , at hos os Alle er der Tider , da hiin saa vigtige og alvorsfulde Gjenstand for Troen ligesom trceder i Baggrunden , medens denne Tids vexlende Spil fcengsler vort <b>Blik</b> og fortrinsviis optager Hjertet . Vi kunne derfor Alle trcenge saa saare til at paamindes og opvarmes ved de store Minder , som denne Fest sigter til at fornye , og Alle trcenge vi til at voxe og befcestes i Troen paa Opstandelse og det evige Liv . Men hvorledes stal denne Tro da vel kunne stadfcestes og befcestes ? Visselig er det til den Ende gavnligt tidt og ofte at forege i sin Sjcel Mindet om Jesu Opstandelse fra de Dooe og at lcegge sig paa Hjertet , at ingen Begivenhed er stcerkerc beviist og stadfcestet end denne . Men ligeoverfor Vantroen , som altid vil trcenge sig frem , ere faadcmne Overveiclser stnndom afmcegtige ; det viser sig at vcere bogstavelig <b>sandt</b> , hvad Jesus lader Abraham sige til deu rige Maud : „ Dersom de Me troe Moses og Propheterne , saa ville de heller Me troe , om en D / 3 d kom tilbage fra de Rige . " Saa stcerk er Tvivlens og Vantroens Magt . Heller Me ville de Slntningsrcekker , som for Enhver , som troer Paa en levende Gud , saa naturligt tilbyde sig , og som synes at det saa klart , at Mennesket maa vcere bestemt til en anden og HMre Leven end denne jordiske og timelige , saasandt der er en Gud til , og hau — som naturligen Mger af selve Gudsbegrebet — er alviis , god og retfcerdig , vcere tilstrekkelige til at holde Stand imod den tvivlende Forstand og det vantroe <b>Hjertes</b> forenede Angreb . Men hvorledes stulle vi da vinde frem til Fasthed i vor Tro og i vort Haab ? Saaledes , mener jeg , at vi Mge det Raad af Frelseren ftrst at give Agt paa , hvad Moses og Propheterne have at sige os , give Agt paa Guds Villie saaledes , som denne umiskjendcligen og uncegteligen er os aabenbaret i vor Samvittighed og i Guds hellige Lov og i Naturlivets og det aabenbarede Ords dunkle Prophetier , eller med andre Ord : saaledes , at vi forsage at adlyde det as Guds Ord , som vi nKes til at erkjende og Me os under , og give dets Virkning Rum i Hjertet . Ehristus har <b>selv</b> anbefalet os denne Vei , fom en Vei , der sikkert leder os til ham ; „ Troe I Moses , saa maae I og troe paa mig ; thi han har strevet om mig , " siger han etsteds , og paa et andet Sted siger han :

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112101017'target=_>Ahlfeld, Friedrich, 1866, Menneskelivet i Guds Ords Lys</a><h4>99</h4>Slige Underverker formaaer alene Guds Magt og <b>Kjerlighed</b> at gjore . Alene i Hans Haand staaer det at give og opholde Liv . Ja , min Herre og Gud , Du drog mig frem af Moders Liv ; Du har beklcedt mig med Hud og KM og gjennemvevet mig med Been og Sener ; Du har stjcenket mig Legeme og Sjel , Dine og Aren og alle Lemmer, Fornuft og alle Sandser , og din Varetegt bevarer min Aand . Der er tre store Overgange i Menneskets Liv . Den fsrste er . Fødselsdagen , da det trceder ud fra Moderslivet til Lyset ; den anden er Daabsdagen , der fuldendes ved Menneskets Opvaagnen til et klart troende Christenliv , hvori det gjennem egen Erfaring gaaer over fra Vredens til Naadens Stand ; den tredie er dets falige Dsdsdag , da det fra Troen gaaer over til Beskuelsen . Alle tre ere indbyrdes beslcegtede ; alle tre ere vosldige Skridt . Naar man fra Mprket kommer md i et klart Lys , saa er dette noget Nyt og Uvant . Naar man fra de kolde og skåldede Fjelde stiger ned i et varmt Land med Vaarluft og Blomsterpragt , saa er Afstanden mellem begge Dele stor . Man kan i Førstningen ikke faae <b>Oine</b> og Sandser vidaabne nok . Og dog ere hine tre Overgange , og dog er Overgangen ved et Barns Fsdsel langt swrre . Fra sit mprke Kammer trceder Barnet ud i en ny Verden . Lyset er det Fyrste , der hilser det . Ved et Barns Fsdsel gaaer Gud den samme Vei , som Han gik ved Verdens Skabelse . Og med dette Lys modtager det tillige en Hilsen fra den Gud , forn det tilhører ; thi Gud er Lys , og der er aldeles intet Morke i Ham . Og dog , neppe har det flaaet Vinene op , fpr det lukker dem til igjen , forn om det vilde sige :  leg taaler ikke at beskue Hans Kledebon , den fsrste Lignelfe af Ham , for hvem endog Seraferne stjule sit Ansigt ; thi jeg er fM i Misgjerning . " Saaledes ligger da Barnet der , udstyret med en guddommelig Medgift , men ogfaa med en mennestelig, med Synd og Skyld . Kunde der komme en Reen af en Ureen ! Nei ikke Een . Hvad forn er fM af Kjsdet , er Kjsd . Barnet er avlet af fyndig Sed , og dets <b>Moder</b> har undfanget det i Synd . Som et Barn af sine Foreldre og Forfedre , ' berer det disses Arvelod indesluttet i sig . Slegtens hele Historie er sammenfattet i det . Men i Serdeleshed ville Fader og <b>Moder</b> gjenfinde sine egne Synder hos Barnet ; ja sine egne Synder ; thi Synden forplanter sig af sig <b>selv</b> , ligesom Ukrud ; Frelsen derimod maa fprst dyrkes ved Naademiolerne . Saaledes er da ethvert Barn en Knup , hvoraf Alt kan fremblomstre . Naar denne Knup bliver fugtet med Himmeldugg og lader den trenge md i sit <b>Hjerte</b> , da kan den blive til en Blomst , hvori Gud og Engle og

<h4>181</h4>maa vente , indtil et fremmed <b>Hjerte</b> , som har mere <b>Kjcerlighed</b> end Moderhjertet , kommer og tåger det til sig . Ve en saadan <b>Moder</b> ! Hun betragter ikke sit Barn som nogen Guds Gave , men som en Byrde , der er hende paalagt mod hendes Villie , som en Anklager mod hendes Synd og Skyld . Hun veed intet as det Ord :  Men naar hun har fpdt Barnet, kommer hun ikke mere den Trcengsel ihu , af Glcede over at et Menneske er ftdt til Verden . " Trengselen lever med Barnet , og derfor vil hun vcere af med det . Dog , vi ville ikke tale mere om disse Mpdre uden Moderhjerte . Men ner til dem grendse de More i de store Byer , som ester Fsdselen sette sit Barn i en af hine Pleie stiftelser for fattige Diebsrn , som dersteds gjerne findes . Naar saadanne Bl > rn begynde at faae lidt Forstand , mpder hellerikke deres <b>Die</b> nogen <b>Moder</b> , men en fremmed <b>Kvinde</b> , som gjor Smaabsrns Opf ^ oelse — vi kunne ikke kalde det Ovdragelse — til et Haandvcerk , et Levebrpd . Moderen har frasagt sig MBen med sit Barn , men dermed ogsaa Glceden deraf . Ikke stort bedre er det i de fleste Tilfcelder , hvori Vk > rn , som man kalder , udscettes til Ovfostring . Om end dette Skridt ofte kan verre en Mgelig Nødvendighet ) , saa er denne dog fordetmeste en Frugt af Synden . Saadanne Mdre skulde kun ret betcenke , hvorledes Synden forgifter al Glede , hvorledes den lpsner <b>selv</b> de inderligste Forholde , saaledes som Barnets til Moderen . Sjelden , sjelden betragter Barnet senere Moderen som sin rette <b>Moder</b> . Denne har jo hellerikke <b>selv</b> betragtet Barnet rigtigt som sit Barn , har ikke med suur Mmc og vaagne Ncetter kjpbt sig en Plads i Barnets <b>Hjerte</b> . Gud havde givet hende det som en Gave , og hun har ikke villet have det som saadan . Men <b>selv</b> bortseet fra disse Afveie , staaer ofte Sorgen i Glcedens Sted ved det stakkels Barns Vugge . Istedetfor Taknemmeligheds- og Glcedesstraaler seer Barnet Kummerens Taarer i Foreldrenes Nine . Istedetfor den fsrste Vuggesang horer det Sukke . Ofte tenke fattige Foreldre :  Barneflokken bliver for stor . Hvem skal fsde den ? Vi kunde med Nod og Neppe skaffe Brsd til dem , vi havde , og nu skjcenker Gud os eet til ! " Mangen Fader har allerede knurret under sin Hustrues Svangerskab og siden seet snurt til sit yngste Barns Vugge . Og dog er dette ikke Andet end Mangel paa Tro , Mangel paa Mod . Du er ikke Barnets sverste Fader . Dets Skjebne styres af Ham , fra hvem alt Fadernavn er i Himlene og paa Jorden . Han har antaget det til sit Barn i den hellige Daab ; Han har ladet det dsbe til sit Navn ; Han har ikke sagt :  Flokken bliver for stor ; jeg vil ikke mere lade noget Barn dstbes til mit Navn , til min Naade og Omsorg . " Nu

<h4>243</h4>saaledes vil det ogsaa gaae med dine Bsrn . Iden tidlige Barndom er Hjertet bIM og Hukommelsen ung . Hvad man da lerer , det indpreger sig dybt og glemmes fordetmeste aldrig . Men ikke nok hermed . Hvad vi lcere i vore fyrste Barneaar , og navnlig af vor <b>Moder</b> , det seer anderledes ud end hvad vi senere lcere . Morgenduggen henger ved det ; Moderkjerligheden med den hele Erindring om disse lykkelige , sorgfrie Aar omsvever det . Naar vi senere treffe saadanne Udsagn og Fortellinger igjen , tale de et ganske andet Sprog end de , vi siden have lcert . Dette ligger ikke i Guds Ord , som om nogen Dccl af samme var kraftigere end en anden . Det ligger i Tiden og Hjertets Modtagelighed. Disse fsrste Udsagn og Fortellinger have staaet Rpdder i hele Livet . Saaledes er det ogsaa med Sceden ude paa Ågeren . Hvad der saaes tidligt om Vaaren , fester dybest Rod . Om ogsaa en anden Ager senere besaaes med samme Sced , modstaaer hun dog bedre Torken og lunner Landmanden med en rigere Afgwde . Hvad der fra fprst af er indprceget i Barnets <b>Hjerte</b> , giver dets Liv dets Retning . Historien kjender en Mand , som allerede i sine tidligste Aar legede med en Kanon . Siden arbeidede han nesten hele sit Liv igjennem med dette Mordredskab . Da er det bedre , at vi tidligt mode Herren ; da kunne vi ikke vride os fra Hans Dragelse, saalenge vi leve . la , om vi end senere fare vild en Stund , om end Livets <b>Sand</b> og Stsv legger sig paa hine Sedekorn , saa er der dog fremdeles Liv i dem , og i sin Tid komme dog de fsrst op. Intet har oftere fort forvildede og i Elendighet ) nedsunkne Syndere paa Omvendelsens Vei end Tilbageblikket paa en gudfrygtig og lykkelig Ungdom og Ihukommelsen af de Stykker af Guds Ord , som de havde lert i sin Barndom . la , nettop de ere for Mange blevne en Kjep og Stav i Dposskyggens Dal . Men hellerikke for Eder Foreldre bliver et saadant Arbeide paa Eders Bsrns Frelse uden Frugt . Kun den Fader og <b>Moder</b> bestoder i Barnenes Vine den rette Verdighed , som har den himmelske Fader staaende bagved sig , som forretter sit Kald i Hans Navn og henviser sine Born til Ham . Da hviler der en Glands af guddommelig Majestet om deres Hoveder , som fremkalder en langt sterre og en langt sandere Lndighed end al blodagtig <b>Kjerlighed</b> og alle strenge Befalinger . Dette er imidlertid en Frugt , som falder mere udåd , medens derimod en anden falder i dit eget <b>Hjerte</b> . Jo eenfoldigere vi tåge Guds Ord , desto sterre Magt har det . Hvo som <b>selv</b> er ret liden og eenfoldig , idet han meddeler sine Bsrn det , hvo som nedlader sig til dem , ligesom Gud Herren i Ordet <b>selv</b> har nedladt sig til os , han modtåger

<h4>348</h4>deres Lceber ! For det Andet tåger Gud ofte Foreldrene bort , for at de ikke skulle forderve sine B ^ rn med sin Vantro og sine Synder . Rige og overdaadige Folks Bsrn have ofte mistet sine Foreldre , for at de ikke skulde drages ind i Verdens Overdaaoighed og Ugudelighed . Gud har bragt dem i et Huus , hvor de have lert at arbeide , bede og ssge Hans Ansigt . Be » rn af fattige Foreldre , men som dog Intet vidste om Frelsen og Frelsens Vei , har Han ofte bragt i et troende Huus eller en Ovdragelfesanstalt . En gudfrygtig Pleiefader og Pleiemoder , en gudfrygtig Forstander eller Forstand erinde have da gjort imod Bsrnene , hvad deres Foreldre ikke havde gjort . De have fsrt dem til Herren . Opfostringsanstalten eller Pleieforeldrenes Huus er blevet dem et nyt Livs Vugge . Mange fattige BjZrn have i en modnere Alder bekjendt for Gud :  Min Fader i Himlene ! da jeg mistede min kjpdelige Fader , fandt jeg Dig . " Og endelig for det Tredie lader Gud mange Vprn blive foreldreløse , for at Menigheden skal blive deres <b>Moder</b> . O , dersom den ogfaa isandhed blev det ! O , dersom den blot stedse i <b>sand</b> Menighedsf ^ lelfe bekjendte :  Det er vore BMn ; vi maae vere dem i Foreldres Sted " ! Men Gud hjelpe os ! hvorledes blive ikke de Foreldreløse ofte behandlede ! Vi vide , at de stakkels B < Zrn i enkelte Menigheder ere blevne satte til formelig Licitation og overdragne til de Mindstbydende . Man har behandlet deres Ovdragelse paa samme Maade som et eller andet übetydeligt jordisk Anliggende , som en Vei , der skulde istanosettes , som en Dam , der skulde ovfpres. Man har ikke bekymret sig om til hvem Barnene kom , hvorledes de blev ovdragne , hvortil de blev brugte . Hvor mange Foreldreløse have ikke maattet sjZge at vekke Medlidenhed for sig <b>selv</b> og sine Pleieforeldre og gaae for Hvermands De » r ! Ofte har Menigheden viist sig som en ugudelig Stedmoder mod de stakkels Be « rn , og i hele Aar have deres Formyndere ikke erkyndiget sig om dem . Naar Foreldrene ose , bsr de nermeste Slegtninger , Foreldrenes Spstende eller andre Paawrende , Fadderne eller Formynderne tåge sig af deres Bprn . Hvad er ellers Slegtskabet til ? Naar et Lam mister sin <b>Moder</b> , give de andre Faar i Hjorden det forladte <b>Die</b> tilligemed sine egne . Nesten blandt alle Folkeslag gaaer der Sagn om at en Hind eller endog en Nlvinde har givet udsatte Bprn <b>Die</b> og er bleven deres Pleiemoder . Ere da ikke I meget mere end de ? Bsr der ikke boe en ganske anden <b>Kjerlighed</b> i Guds Bprn end i de stakkels fornuftslsie Skabninger ? I ligegyldige og ukjerlige Menighedsforstandere og Slegtninger , og I Mammonstrelle , som udstrekke Eders Haand ester de Foreldrenes Gods ,

<h4>702</h4>Herre Jesus Christus , lad aldrig Brud og Brudgom , lad aldrig Mand og <b>Kvinde</b> glemme , at Du er deres fyrste Brudgom . Du har elsket os med en evig Kjærlighet ) og draget os med Miskundhed . Du har beilet til os med en <b>Kjcerlighed</b> , hvis Lige aldrig er opkommen i noget Menneskes <b>Hjerte</b> . I den hellige Daab satte Du Trolovelsesringen paa vor Finger . Med din tilgivende , sagende , skaanende og bcerende <b>Kjcerlighed</b> vil Du drage os ftedse ncermere til Dig . O Herre , giv os Naade til at lade os drage , til med hver Dag at blive stedse ncermere forenede med Dig . Giv ethvert trolovet Par , at de Begge maae trolove sig stedse inderligere med Dig , blive uadskillelige fra Dig og saaledes ogsaa vcere indbyrdes forbundne i een og samme Tro , een og samme Frelser , een og samme Netfcerdighed og HelliggMelse og een og samme Vandring til det Jerusalem heroventil . Ak , Herre , lad din Forbarmelse og Saligheden i Dig kaste sit <b>Blik</b> , sit Morgenskjcer i enhver Forlovelsesstand, for at det ogsaa kan stinne gjennem 2Egtestanden og ovklare Huset tilligemed dets Glceder , dets Mpisommeligheder og Trcengsler . Det forunde Du af Naade vore forlovede Par ! Amen . Hpit cerer Guds dyrebare Ord Forlovelsesstanden . I vor Text sammenligner Gud sin Glcede over sit troe udvalgte Folk med en Brudgoms Glcede over sin Brud . Vor Herre og Frelser kalder sig <b>selv</b> Brudgommen og Menigheden Bruden . Johannes den Dpber er Brudgommens Talsmand og Brudeus Fsrer . De ved hans Prcediken ovvakte og i Jordan dchte Sjele forer han til Herren , Brudgommen. Herren sammenligner de Dage , Han tilbringer her paa Jorden , med den Tid , Brudgommen er hos Bryllupsgjcesterne. Brvluvsgjcesterne skulle ikke ssrge , saalcenge Brudgommen er hos dem . la , hele den Tid , hvori Kirken er stridende , hvori den forventer Herrens Gjenkomst for at fuldende sit Frelfervcerk , sammenlignes med en . Forlovetsesstand.

<h4>754</h4>Ved Bryllupsmaaltidet maa Borobmnen ikke mangle . Har den end lcenge sovet i en Familie , saa er dog dette en Dag , paa hvilken den bpr vcekkes . Og har man aldrig bedet i en Familie , saa maa man dog paa en saadan Dag fjSle , hvor fattig , hvor tom , hvor kold man scetter sig tilbords, naar man ikke har takket Herren og ikke anraabt Ham om Hans Naade i Fremtiden . Endnu paaligger det ester almindelig Vedtcegt Brudens Fader at forrette Bordbmnen paa denne Dag . Hans Bsrn og Gjcester spise endnu ved hans Bord . Forsømmes Bpnnen paa denne Dag , — slutter ikke Pigens Liv dermed i Forceldrenes Huus , saa vil vel heller ikke Livet i hendes eget Huus blive begyndt dermed . Man undlader da at medgive Bsrnene en as de kosteligste Bygningsstene i Familielivet . Ved Bryllupsmaaltidet er det en stor Glcede at see begge det unge Pars Familier sammensmeltede til Eet i Herren . I Midten sidder det unge LEgtepar , og til Hmre og Venstre Fcrdrene og Mpdrene . O , hvor ncer ere ikke disse nu bragte hinanden ! Deres <b>Kjcerlighed</b> og Banner hvile jo nu paa Barnenes fcelles Huus . Og efterdi Mand og <b>Kone</b> ere Eet , bor det unge Par ogsaa staae i et <b>sandt</b> Barneforhold til hinanoens Forceldre . Gid enhver ung Mand og ung <b>Kone</b> ret af Hjertet vilde kalde sin ZEgtefcelles Forceldre Fader og Moder! Gid Forceldrene ogsaa vilde betragte sit nye Barn med ret faderlige og moderlige Melser ! Det gjor en <b>Hustru</b> ondt , naar hun vel har vundet sin Mands , men ikke tillige hans Forceldres <b>Hjerte</b> . Da blceser det hende saa koldt impde i hans Familie ; hun forekommer sig <b>selv</b> som en Kvist , der ikke er bleven rigtigt indpodet . Sorg og Suk gaaer da allerede tidligt gjennem hendes <b>Hjerte</b> . Herren give , at en oprigtig Forening mellem begge Familier allevegne maa finde Sted ! Dog nok herom . Vi maae endnu omtale nogle Andre , som ogsaa have sine Fordringer til Brylluppet . Overalt i Christenheden , hvor man har feiret en rigtig Glcedesfest , har man ogsaa betcenkt de Fattige . Glceden uovioer Hjertet . Glceden ihukommer ogsaa dem , som ikke glcede sig paa den Dag . Gud Fader har indbuot Fattige , Halte , Krsblinger og Blinde til sin Ssns Bryllup . Vi ere <b>selv</b> saadanne Gjcester . Saa ville vi da ikke glemme de Fattige i vor Glcede . Paa Landet stal det tidligere have vceret Skik , at man hensatte et eget Fad , hvorvaa man lagde Noget af hver Ret til dem . Dette kaldte man Guds Dccl , den Dccl , Han ved de Fattige afhentede til sig . Dette er vel nu gaaet af Brug ; men man kan dog endnu give Gud Hans Dccl i Penge og andre Gaver . Ogsaa de Fattige skulle bede for det unge Par og takke Gud for

<h4>795</h4>ncermere end en Vens med sin Ven , en Broders med sin Broder , et Barns med sine Forceldre ! Ak , Herre , Herre ! ZSgtefolk stulle vcere Eet > stulle vcere eet Kjso . O , gM Du ret dem enige ! Bed Du for ethvert christeligt LEgtepar . Lad den Hellig-Aand arbeide paa dem for at drcrbe deres Selvraadighed og Egensindighed , deres Hjerters Kulde og Hovmod . Drag dem Begge md i ' dit Ord og din Aand . Lad det Ord fra Alteret :  Hvad Gud har sammenfpiet , stal intet Menneske adskille , " stedse mere og mere blive <b>Sandhed</b> hos dem . Herre ! hellerikke den gamle Adam med sin Egenvillie og Trodsighed b ^ r adskille dem . Hellerikke i Hjerterne bpr det komme til nogen Adstillelse og Kulde . Giv tvertimod Naade til at ethvert Par maa blive stedse fastere sammenfsiet i Dig , maa faae Dig og hinanden stedse kjcrrere og hverken kunne stilles fra Dig eller hinanden . Giv dem dertil Alvor , Aarvaagenhed , BM , Forsigtighed , Idmnghed , og Taalmodighed . Lad dit Ord boe i ethvert Huus og Dag for Dag blive kraftigere i Hjerterne . Formedelst din hellige Fødsel , formedelst din Trolovelse med vor Slcegt ville Du stjcenke os denne Naade ! Amen . Ved Alteret er HEgteparret viet og saaledes efter Guds Orden blevet Eet . Begge Parter have <b>selv</b> valgt hinanden og erklceret , at de elske hinanden , ja elske hinanden hsiere end noget andet Menneske . De have . givet hinanden det hellige Lpfte at ville vandre Pilegrimsveien gjennem dette jordiske Liv i Fellesstab og dele Sorg og Glcede med hinanden. De boe i Huus sammen ; de staae iet inderligere og fortroligere Forhold til hinanden indbyrdes end til andre Mennesker . Og dog er der mangfoldige Par , hos hvilke det aldrig kommer til nogen <b>sand</b> Enighed . Vi ville nu slet ikke tale om de ZSgtefolk , som kun en kold Beregning har fprt fammen , i hvilket Tilfcelde det saaledes egentlig ikke er Personerne , men deres Midler , som ere blevne sammenviede. Men <b>selv</b> mellem dem , som have rakt hinanden Haanden af eget Valg og naturlig <b>Kjcerlighed</b> , hersker der ofte ingen Eendrcegtighed . Den <b>Kjcerlighed</b> mellem Forlovede, som svcevede i Skyerne , den drpmmende og svcermeriske <b>Kjcerlighed</b> , der heller vilde djZe end give Slip paa den Elskede , hvor snart kjølner ikke ofte den ! Mikal , Sauls anden Datter , havde elsket David . Den unge Helt var en Mand efter hendes <b>Hjerte</b> . Hun cendsede ikke sin Faders Vrede ; hun blev Davids <b>Hustru</b> . Da Savl vilde drcebe ham , frelste hun hans Liv . Da Morderne trcengte md i Huset , havde hun allerede hjulpet David til at flygte . Ved at lede i Sengen fandt Fienderne et Afgudsbillede , som Mikal havde lagt der istedetfor ham . Da David omsider var

<h4>828</h4>Mand og <b>Kone</b> ere for Fremtiden ikke To , men Eet ; følgelig ere ogsaa deres Forceldre en fcelles Eiendom . Navnet  <b>Mor</b> " bpr ikke vcere nogen blot Talemaade i Svigerdatterens Mund . Denne <b>Moder</b> har hun at takke for sin Mand ; hun har baaret ham under sit <b>Hjerte</b> , bragt ham til Verden med Smerte og opdraget ham med Msie . Meget af det , forn den unge <b>Kone</b> elsker hos ham , har nettop denne <b>Moder</b> indplantet ham . Har Svigermoderen sine haarde og underlige Sider , saa skal Svigerdatteren kun ret ofte gjentage Navnet , , <b>Mor</b> . " Om hendes egen <b>Moder</b> var af den samme Sindsbeskaffenhed , vilde hun ikke opscette sig mod hende . Her er ogsaa en <b>sand</b> <b>Moder</b> ; thi Wgteskabet er en <b>sand</b> Forening . Derfor bisr <b>Konen</b> ogsaa vel vogte sig for at drage sin Mands <b>Hjerte</b> fra hans <b>Moder</b> . Deraf vil en <b>Hustru</b> aldrig hoste Velsignelse . Hvo som giver Anledning til Overtredelse af det fjerde Bud , arbeider med det Samme paa Overtredelsen af det sjette . Budene ere ti Grene paa een Stamme . Naar en af disfe Grene visner , gaae ogfaa de andre no ; thi da er hele Treet sygt . Ingen <b>Hustru</b> maa indbilde sig at kunne fastholde den Mands <b>Kjcerlighed</b> , som hun har gjort utro mod sin <b>Moder</b> . Hvo som lunner den <b>Kjcerlighed</b> med Utak , der har fulgi ham fra Livets fyrste Morgen til denne Dag , han vil endnu snarere vende den <b>Kjcerlighed</b> Ryggen , som Mft senere er indtraadt i hans Liv . Den , som er bleven utro mod den medfpdte <b>Kjcerlighed</b> , bliver det ogsaa mod den cegteskabelige. For det Tredie bor en ung <b>Kone</b> vel vogte sig for at beklage sig over sin Svigermoder udenfor sit Huus . <b>Selv</b> til sin egen <b>Moder</b> og sine egne Ssskende bor hun ikke sige noget Ufordeelagtigt om hende . Har hun Noget at klage over , kan hun ikke holde det hos sig <b>selv</b> , saa kan hun klage for Gud Herren .

<h4>834</h4>DBden . Hvad blev Fplgen af denne yomyge , udholdende Kamp ? Svigermoderen lod i den Grad sin Bitterhed og sine Fordomme mod Monika fare , at hun en Dag opfordrede sin SM til eftertrykkeligen at revse de sladderagtige Tjenestepiger , der havde besmt hendes Svigerdatter . Patricius gjorde det , og hans <b>Moder</b> erklcerede derhos , at den samme Lm kunde Pigerne herefter vente sig , hvergang de bragte Sladderposter til hende om hendes Svigerdatter ; thi hun havde nu overbeviist sig om at denne var en ganske fortreffelig <b>Kone</b> , og at Alt , hvad de havde beskyldt hende for , var lutter ondskabsfulde Ovdigtelser . Fra den Tid af herskede der en hjertelig god Forstaaelse mellem begge Koner, og naarsomhelst Monika led nogen Uret , optraadte den Gamle til hendes Forsvar . De samme Vaaben staae endnu den Dag idag til enhver Svigerdatters og Svigermoders Raadighed . Jesu Christi Rustkammer er altid aabent . Jo Mere vi tåge deraf , desto mere glceder det Ham . Hvad Monika kunde bringe det til med den afrikanske Hedningkvinde, det vil du , kjerre Lceserinde , endnu snarere kunne bringe det til med en Christenkvinde . Den af Herren fsdte <b>Kjcerlighed</b> , Idmyghed , Taalmodighed , Venlighed og Tjenstagtighed vil dog tilsidst vinde Seier . Deiligt er det , naar en livstrcet <b>Moder</b> velsignende udbreder sine Hcender over sin Datter og siger :  leg har baaret dig under mit <b>Hjerte</b> ; jeg har opdraget dig med Bpn og Mmc , og du har linnet mig derfor som et <b>kjcerligt</b> Barn . Du har i alle dine Dage hast det fjerde Bud for <b>Die</b> ; din <b>Kjcerlighed</b> og Troskab har vceret min Alderdoms Stav , Lys og Varme . Jeg gaaer nu hjem ; du bliver endnu her . Gid det fjerde Buds Velsignelse maa hvile paa dig , saalcenge du vandrer herneoen ,  paa det dine Dage maae blive mange , og det maa gaae dig vel i Landet ! " Og naar din Time kommer , saa Mg ester mig til min Herre og din Herre . Amen . " Men endnu deiligere er det , naar en Svigermoder af et taknemmeligt <b>Hjerte</b> siger til sin Svigerdatter :  Du indvodede Kvist har vceret mig en <b>sand</b> , cegte Datter . Du har deelt din <b>Kjcerlighed</b> mellem dine Forceldre , din Mand og mig og dog rigeligen betcenkt Enhver af os . Du bar fundet dig i mine Skrsbeligheder , og min Alderdoms Luner have ikke formaaet at udtsrre din <b>Kjcerligheds</b> Kilde . Du har baaret mig paa bedende , trofaste , hjelpsomme Arme ligetil denne sidste Stund . Gid du maa faae din Lm herfor . af den Fader , som har en Velsignelse til alle sine Bsrn ! Han velsigne dig i Bprn og Bsrnebsrn ! Han opholde dig i Troen indtil din Ende ! Han lade sine kjcerlige Engle boere baade mig og dig hjem til Ham ! " Tro mig , kjcere Lceser ; ogsaa Svigerforceldrenes

<h4>996</h4>Forbarmelse . Han er jo <b>selv</b> den barmhjertige Samaritan , som forbinder den , der er falden blandt Rpvere , lsfter ham op paa sit Dyr , fprer ham til Herberge og pleier ham . Fra Ham stammede da ogsaa den christelige <b>Kjerlighed</b> i den apostoliske Menighed , som fjZrst indrettede en ordentlig Fattigpleie . Ham var det , som gav Paulus i Sinde at samle en Almisse blandt de rige Menigheder i Makedonien og Grekenland til de as Hungersnød hjemspgte Christne i Ipdeland . Ham var det , som indstjsd Tabitha i Byen Lydda at gjore Kjortler og Kleder til Enkerne. Ham var det , som bevcegede Onesiforus ' s <b>Hjerte</b> , om hvem den fangne Paulus skriver :  Han har ofte vederkveget mig og skammede sig ikke ved min Lenke ; men da han var i Rom , sßgte han saare flittigt efter mig og fandt mig . Herren give , at han maa finde Barmhjertighed hos Herren paa hun Dag ! " Dog , hvis man vilde udskrive alle de Steder i Guds Ord , der handle om Barmhjertighed , maatte man udskrive hele den dyrebare Bog . Ikke <b>sandt</b> , du unge ZLgtepar ? — ligesom Herrens Barmhjertighed er opgangen over Eders Huus , saaledes bsr ogsaa Eders Barmhjertighed skinne ud deraf til Eders fattige Brpdre . Men til hvem bpr den komme ? Vi betMe ikke at spsrge lcenge . Herren siger :  I have altid Fattige hos Eder . " Nu gives der vistnok Fattige , som ogsaa den hellige Skrift frakjender al Fordring paa Barmhjertighed .  Dersom Nogen ikke vil arbeide , han bpr ikke heller ede / " Hungeren skal drive ham til Omvendelse og Arbeide . Hvor vi understøtte Synden med vore Gaver , der begaae vi en Synd ved at give . Fattige Familier , som alvorligt strebe at tjene Herren og at cede sit Brso i sit Ansigts Sved , Syge , Gamle , Enker og Faderlose ere de Altere , hvorvaa Barmhjertighed ens Offere be < r lcegges . Der gives Familier , Syge og Elendige , som trcenge til stadig Understøttelse . Men der indtrceder ogsaa andre Tilfcelder . Ofte staaer den bitreste Nsd allerede ftrax ved de nyfsdte Bsrns Vugge ; her bjZr Forbarmelsen stille sig ved Siden as denne , for at ikke den nye Himmelborgers fprste Blikte stulle falde paa Foreldrenes Taarer . Naar en Familieforsørger dper , skal den christne Menighed i Gjerningen vidne om at baade Herren og Hans <b>Tjenerinde</b> Menigheden endnu leve . Naar fattige Bsrn stulle staae til Konfirmation , overstiger det ofte Foreldrenes Evne at forskaffe dem en ny Kledning , hvori de kunne tenke paa det Saligheoens Kledebon og den Retferdighedens Kappe , fom Herren ifprte dem i Daaben ; da bpr den christelige <b>Kjerlighed</b> oplade sin Haand for at bringe denne store Naade i levende Erindring <b>selv</b> hos den Fattigste.

<h4>1035</h4>De Fattige derimod gaae indad den og lade sig gjsre rige af Herren . Istedetfor at knurre over din Fattigdom bpr du heller takke Herren for at Han har bygget din Hytte saa ncer ved Frelsens Port , og at Hans Indbydelser lyde saa klart og tydeligt derind til dig . Hold du kun fast ved din Frelser . Vederkvceg du dig og Dine med Guds gode Ord . O , hvor deilige blive ikke Aftenerne i den fattige Hytte , naar Faderen sidder og lceser HM i <b>Sandhedens</b> evige Ord , og <b>Kone</b> og Bprn begjcerlige og glade lytte derpaa ! Da opfyldes den lille Menighed med den Hellig-Aand ; Hjerterne udvide sig ; den dunkle Lampe broenoer klarere ; Fattigdommen glemmes ; I fryde Eder i Herren og Hans Styrke ; Guds <b>Kjcerlighed</b> er udpst over Eder ; I ftle og ane , at I have en usigelig ! rig Arv . Lad derhos Intetsomhelst stjcele de velsignede Smdage fra dig . Du veed , at de fremfor alle andre Dage ere rige paa Herrens Naade . Paa dem skinner det evige Livs Morgenrøde md i din Hytte ; de ere dig et Pant paa den store Sabbat , paa den store Hvile hos Gud . Semdagen udsoner dig med alle andre Steender . Alle Steender i Menigheden tMe det samme Ords Forkyndelse , have den samme Frelser , ydmyge sig for Gud i den samme Syndsbekjendelse , blive retfcerdiggjorte ved den samme Christi Fortjeneste , ere deelagtige i det samme Christi Legeme og Blod og have det samme evige Livs Haab . Da gaaer den Opstandnes Wst gjennem Menigheden og raaber :  Fred vcere med Eder ! I mig er der hverken Mand eller <b>Kvinde</b> , hverken Isde eller Grceker , hverken Trcel eller Fri , hverken Fattig eller Rig . De , som ere i Christo , ere nye Skabninger . " Med Hensyn til dit Huusvcesen , da vcer tilfreds med hvad Herren skjcenker dig . Ingens Liv bestaaer i hans Gods , i det , at han har Overflod . Eier du Admnghed og Beskedenhed , saa er du ligesaa rig som den Rige . Den Rige kan hellerikke mere end spise sig moet ; han kan hellerikke boe i mere end eet Huus ; han bcerer hellerikke mere end een Kloedning ad Gangen ; han sover hellerikke i mere end een Seng ; han faaer tilsidst hellerikke mere end een Ligkiste og een Grav . Kan du undgaae det , saa laan ikke ; indstrcenk dig heller , saa meget du kan . la , bed heller et christeligt <b>Hjerte</b> om en Understøttelse , fpr du stal gaae hen og laane . Laan gM Lsgnere og letsindige Mennesker . Naar Dagen kommer, at man skal betale tilbage , og man ingen Penge har , undseer man sig for at tilstaae dette for sin Giceldner . Af Hovmod bliver man da en Lsgner , tåger Skade paa sin Samvittighed, synder <b>selv</b> og frister ogsna den Anden til Synd .

<h4>1133</h4>gaaer den Formaning :  Vcerer gudfrygtige ; studerer flittigt , og beder for os herhenne ! " I den samme Tone skrev deres Fader til dem og sendte dem blandt Andet Luthers Katekisme i det hebraiske , grceste , latinske og tydske Sprog . Hendes Anste , at det maatte forundes hende , endnu engang at vederkvege sig med sin Mand — men hellerikke uden ham — i Friheden og Hjemmet , gik ikke i Ovfyldelse . Hun dpde iFangenskab 1594. Hendes sidste Bpn til de om hendes Seng forsamlede Tjenere gik ud paa at de ester hendes D » zd vilde behandle hendes Mand med fordoblet Troskab og Omhyggelighed og vcere ganske det Samme for ham , som hun <b>selv</b> saa gjerne havde villet vcere . Dette var en fyrstelig Korsdragerinde . Nu stulle vi fortcelle om en af borgerlig Stand . I en By i det sydlige Tydskland levede der en Borger , som havde faaet Vcertshuset kjoerere end sit eget Huus . Aftenerne , der hos os ere ligesom skabte og dannede for Familielivet , tilbragte han udenfor sit Hjem i en Kreds af ryggeslpse og lystige Kammerater . Dette fandt hans <b>Kone</b> sig i med stille Sorg , uden Trcette og stmende Optrin . Naar han kom hjem , var hun ligefuldt venlig mod ham . Hertil bad hun daglig Herren om Styrke og ncerede det Haab , at hun paa den Maade snarest skulde drage sin Mands <b>Hjerte</b> fra den steile Afgrund op paa den hellige Klippe . Engang , da han seent paa Natten var sammen med sine Svirebrødre , kom Talen paa deres Koner . Han roste sin Hustrues Sagtmodighed og paastod , at hvis de nu ved Midnat gik hjem til ham og forlangte , at hun skulde lave Mad , dcekke Bordet , og bevcerte dem , saa vilde hun gjpre det uden Knur . De Andre loe ; men han blev ved Sit . Det kom til et Vceddemaal mellem dem og derpaa til et Forftg . Den , trofaste <b>Hustru</b> stod op , lavede Mad og opvartede dem ved Bordet , just ikke med Glcede , men dog med Vcerdighed og ftmmelig Anstand . Da blev Selskabet ilde ved ; Maaltidet vårede ikke lcenge ; de Fremmede gik , og Manden blev alene tilbage . Slagen af sin Kones tjenende <b>Kjcerlighed</b> , spurgte han hende , hvor hun fik Kraft fra til saadan Troskab . Hun svarede ham :  Dit Liv er et Liv i Verden ; Verden forgaaer , og dens Lyft ; men bag Verden ligger der en usalig Evighed for alle haardnakkede Foragtere af Gud , fslgelig ogsaa for dig , saaledes som du hidtil har levet . Med den Elendighed for <b>Die</b> , forn du gaaer inwde , fsler jeg Trang til at gMe dig de faa Aar , som du endnu har tilbage at leve i her paa Jorden eller som jeg endnu stal leve med dig , saa venlige og behagelige som muligt . Dette er Grunden til at jeg fgier dig i Alt , og Herren giver mig Kraft dertil . " Dette gjorde Ndstaget . Dette var den sidste Aften , Manden havde

<h4>1229</h4>Naar du gjor dig noget mere fortrolig med denne herlige <b>Sandhed</b> , saa staaer du alle Born i din Menighed ncer med dit <b>Hjerte</b> . Du maa bede for dem , vaage over dem og rcekke dem en hjcelpsom Haano , hvor du kan . De paa sin Side staae da ogfaa dig ncer . Hvo som imodekommer dem med et faderligt eller moderligt <b>Hjerte</b> , for den have da ogsaa de et barnligt <b>Hjerte</b> . Nu kan vistnok Ingen rcekke Alle Haanden paa samme Maade . Men i enhver Menighed gives der Huller , som den hjcelpende <b>Kjcerlighed</b> bor tilstopve . I enhver Menighed gives der Born , som have mistet den ene af sine Forceldre eller dem begge . Hvis <b>Kjcerlighed</b> have disse iscer Krav paa ? Paa deres , som Gud ikke har skjcenket Born . I det ene Huus er der ingen Born , i det andet ingen Forceldre . Hvad er da klarere end at Gud Herren vil udfylde Mangelen af den ene Dccl med den anden ? Men har en Familie med Trostab taget sig af forceldrelose Born , saa behover den vist ikke i Alderdommen at klage over Eensomhed og Tomhed . Erfaringen viser noksom, at der ikke er taknemmeligere Mennesker til end forceldrelofe Born , som have gjenfundet Fader og <b>Moder</b> . Men ville I virkelig tåge et fremmed Barn til Eder som jert eget , saa tåger det i saa ung en Alder som muligt . Dets Opdragelse vil da falde Eder langt lettere ; det slutter sig da langt inderligere til Eder , og I blive virkelig Fader og <b>Moder</b> for det . Halvvoxne Born blive ofte bestandig fremmede i Huset , om man end behandler dem med aldrig saa megen <b>Kjcerlighed</b> og Trostab . Vel pleie Wgtefolk <b>selv</b> efter at vcere komne temmelig langt ud i Aarene fremdeles at leve i det Haab , at Gud nok endnu vil velsigne dem med Afkom , og de opscette derfor fra Tid til anden et saadant Skridt . Deres Haab kan viftnok gaae i Opfvldelse ; men Gud vil dog snarest vise Barmhjertig h ed mod dem , som <b>selv</b> have viist Barmhjertighed . For ikke lcenge siden Horte jeg en gammel barnlos <b>Kone</b> beklage sig og sige :  leg og min salig Mand have begaaet en stor Forsommelse . Vi burde allerede for femogtyve Aar siden have taget et stakkels forceldrelost Barn til os . Da vi indsaae vor Feil , var det for feent . Da var vi for gamle til at opdrage et lidet Barn , og havde vi taget et storre til os , vilde det strar have gjort sig Tanker om det Lille , vi eiede ; det kunde let have elsket vor Formue hoiere end os . Jeg meerker godt , at alle de , som i senere Tider have villet slutte sig til os , have vceret ledede af et eller andet Bihensyn , og jeg foler ret tungt , hvor eensom jeg staaer <b>selv</b> i min ncermeste Kreds . " Her kan man med Eandhed sige :  Hvad du gjor , det gjor snart ! " Og da give Naadens Gud dig den samme Glcede af et saadant

<h4>1535</h4>Til alle Tider have Menneskene fsrget over sine Afdsde . Abraham begrceder sin trofaste <b>Hustru</b> , som havde fulgt ham paa hans mpisommelige Vandringer fra Kaldcea til Mesopotamien, til Kana ' ans Land og ZEgypten . Josef begrceder sin gamle Fader , Jakob , og tilligemed ham holde et stort Klagemaal ved hans Grav . Israels Bprn begrcede Moses i tredive Dage paa Moabs pde Marker . Da den gudsrygtige Kong losias , den sidste vcerdige ZEtling af David paa hans Kongethrone , var falden ved Hadadrimmon i Megiddos Dal , holdt Folket en Beklage over ham , som gjenlyd lcenge bagefter , idet ingen Konge som han senere opsteg paa hans Fcedres Throne . Gaae vi over til det nye Testament og trcede md i den afdsde Lazarus ' s eller i lairus ' s Huus eller mpde Ligtoget i Ncerheden af Nam , saa lyde Klagemaal os overalt imooe . Og det Samme har vceret Tilfceldet hos alle hedenske Folkeslag ; <b>selv</b> hos de dybeft sunkne af dem ftndes der Spor af Sorg og Klage over de Afdpde . Denne Sorg bor ogfaa vedblive i Christi Kirke . Det maa vcere nogle letsindige Mennesker , som kunne staae ved sin Faders eller Moders Dodsleie , Ligkiste og Grav uden nogen smertelig Følelse af hvad Gud i dem havde skjcenket dem og af den <b>Kjcerlighed</b> , som de med dem miste her paa Jorden . Det maa vcere en elendig og letsindig 3 Egtemand , som paa nordamerikansk Viis allerede ved sin forste Hustrues lordfcestelfe kan overveie , hvem der skal blive hans anden <b>Kone</b> . Ved vore Ncerbeslcegtedes Dyd taler Gud Herren saa kraftigt og indtrcengende til os , at denne Tale nok bor gjenlyde en god Tid bagefter og gjsre os vort store Tab fsleligt . Vi ville nu for det Fyrste gjore os felv Rede for hvem vi ssrge over . Vi pleie at sige :  leg sorger for min Fader, min <b>Moder</b> , min <b>Kone</b> , min Mand , mit Barn , min Broder , min Spster . " Men dette er kun i visse Tilfcelder <b>sandt</b> . Ere de vandrede bort i Vantro og Ugudelighed , have de lige til sin sidste Stund vceret hildede i Satans Lcenker , da maae vi vistnok sorge for dem ; thi saavidt vi sorstaae , ere de da dode en dobbelt Dsd . Ak , da er vort <b>Hjerte</b> sønderknust ; da udgyde vi bitre Taarer ; da kan i Grunden heller Ingen trpste os . Og Sorgen bliver saa meget dybere, som vi jo ved enhver saadan Leilighed maae sige til os <b>selv</b> :  Du har ikke arbeidet paa denne Sjel med al den christelige <b>Kjcerlighed</b> og Troskab , som du burde . " Denne Sorg ophprer ikke , forend vi have faaet alle vore Synders Forladelse og , fuldendte i Herren , lcere at betragte Historien , altsaa tillige de Fortabtes Historie , med den guddommelige <b>Sandheds</b> , Retfcerdigheds og Helligheds Vie . Er derimod Nogen hensovet i Herren , saa Mge vi ikke for ham , men

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011061608052'target=_>Wexels, W.A., 1846, Forklaring over Det nye Testamentes Skrifter</a><h4>1068</h4>dant Mere , ved " Drukkenstab og Fraadseri " til alslags Umaadelighed, o . s . v. — " De , som gjore Saadant d . e . som saaledes tjene Kjodet , stulle Me arve Guds Rige ; " sml . 1 Cor . 6,9 . 10. Eph . 5,5 . De staae naturligviis , som übodfcerdige , Synden elstende Syndere , aldeles udenfor Naaderiget og kunne saaledes Me heller arve Herligheds-Riget , Me dele Guds Boms Lod ; deres Arv er meget mere Dod og Fordcervelse ( 6,8 . Rom . 8,13 . Aab.21 , 8. ; 22,15 . ) , og denne maae da end vissere komme over dem , der " pacckalde Christi Navn og Me ville afstaae fra Uretfcerdighed , " der fra Aandens vende tilbage til Kjodets Tjeneste ( Matth . 7,23 . Ebr . 2,2 .3 . 2 Pet . 2,22 - 24 ) . Denne Tilbagevenden er jo det Samme som Affald fra Naaden, hvis retfcerdiggjorendeVirkning st and ser , nåar dens helliggjorende Virkning modstaaes , ved hvilken vi stulle blive mere og mere Mede til at see Gud , leve i hans Samfund , og stadfcestes i vort Kald og Udvcelgelse . Sml . 2 Pet . 1,10 .11 . — " St . Paulus lcerer her det Samme , som Christus har lcert i Matth . 7,16 .17 . , nemlig at Gjerningerne eller Frugterne noksom vise , om Trceerne ere onde eller gode , d . e . om Mennestene folge Kjodet eller Aanden ; som vilde han sige : Paa det at Me Nogle iblandt Eder stulle forevende , at de Me forstaae , hvad jeg nu har talt om Kjodets og Aandens Strid , saa vil jeg stille Eder for <b>Oie</b> , forst Kjodets Gjerninger, af hvilke <b>selv</b> de Ugudelige erkjende en Deel , derncest Aandens Frugter . Der vare nemlig Hyklere blandt Galaterne , ligesom paa ncervcerende Tid , hvilke foregcwe , at de vare fromme og gode Christne , roste sig meget af " Aand " og kunde mcmgfoldigen og herligen snakke om Troens Lcerdom , men vandrede Me des mindre ei i Aanden , men i Kjodet , og fuldkommede dettes Lyster ; derfor lader Paulus dem her offentligen vide , at de Me vare de Folk , som de udgave sig for " ( Luther ) . V . 22. I Opregnelsen af Aandens Frugter eller det gjenfodte indre Menneskes Vcesen og Yttringer ncevner han iscerdeleshed dem , der danne en Modscetning til hiin Bitterhed og Uenighed , som begyndte at faae Indgang i Menigheden , og som derfor besynderlige » burde lcrgges paa <b>Hjerte</b> . Forst scetter han <b>Kjcerligheden</b> , som <b>Moder</b> til alle de ovrige , ja til

<h4>2759</h4>V . 2. 3. De her omtalte christne <b>Qvinder</b> vare rimeligviis Diakonisser ( Rom . 16,1 . og Forkl . dertil ) eller i anden Stilling tjenende i Menigheden . Personerne <b>selv</b> er os übekjendte , som og Anledningen til deres Usamdrcegtighed . Denne maa vistnok vcere gaaet saa vidt , at den var bleven aabenbar og til Anstod for Menigheden, hvilket ogsaa af hiin deres formodentlige Stilling er let forklarligt ; men at de dog har havt et Sind , christeligt i sin Grund , og som altsaa vilde gjore dem modtagelige for Syndens Erkjendelse og beredvilligc til Enighed , nåar <b>Sandheden</b> blev klart og alvorlig ! forelagt dem i <b>Kjcerlighed</b> , maae vi flutte , saavel af Apostelens egen Formaning og hvad han lcegger sin retsindige Medbroder paa <b>Hjerte</b> Sagen vedkommende , som af det Vidnesbyrd , han giver hine <b>Qvinder</b> . " At vcere enige " ; egentlig staaer der : " at mene det Samme " , ligesom i 2 , 2. , saa at Talen ikke blot er om en ydre , men om en i Sindets , i Meningens og Villiens fcelles Retning grundet Enighed . Og denne fcelles Mening og Villie betegnes ved Tillcegger " i Herren" som christelig , som den , der stal gaae ud fra og vevcege sig i Samfundet med Christo og have hans Villie , hans A3re , for Vie ; og hvad stulde ogsaa mere virke til Enighed end <b>Blikket</b> paa ham , den <b>Kjcerlighedens</b> og Forligelsens Herre , hvis Gjerning var at nedbryde enhver Tvistens Skillevceg og stifte Fred , evig Fred , paa <b>Sandheds</b> Grund , <b>selv</b> af Hjertet tilgivende Alt , hvad der mod ham var syndet ! See forovrigt 2 , 2 - 4. og Forkl . dertil . — Ved at gjentageOrdet " jeg formaner " til enhver iscer af de to Uenige , lcegger Apostelen baade Vcegt paa Formcmingen , og henforer denne saaledes til Begge , at den ene ingen Anledning faaer til at mene sig mindre tilsigtet end den anden eller mindre trcengende til Paamindelse . De stilles i den Henseende lige , og det antages da med Grund , at Skylden har vceret paa begge Sider . — Hvo " den retsindige Medbroder" er , hvem Paulus paalcegger at antage sig hine to Menighedens Tjenerinder til Bilceggelse af den mellem dem vcerende Tvist , kan ikke afgjores . Nogle har ikke uden Grund meent , at det var Epaphroditus <b>selv</b> , Ovcrbringeren af Brevet

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015082448274'target=_>Landstad, M.B., 1871, Kirkesalmebog</a><h4>281</h4>ham med Lcengsel <b>sand</b> , Hans <b>Kjcerlighed</b> at vcelge nd , I Acmden brcendende for Gudl 12. Elsk ham , som eder elskte saa , Gjor ham igjen hvad I formåa , Med gode Lofter fra ham gak , Velsigner ham og siger Tak . 13. O himmelske Miskundheds Mand , Du Lwsens Haab i Dodens Land , Du Naade-Kilde fuld af Trost , Du sode Glcede i mit Bryst ! 14. Ieg taler med uvcerdig Mund , Men tie kan jeg ingenlund , Min <b>Kjcerlighed</b> den tvinqer mig , For Hjertet er saa ftlldt af dig . 15. Med dem , dig smage , saa det ster , De lcenges altid ester nieer , Du aldrig dem af Minde gaar , Til dig al deres Attråa staar . 16. Jesu , Guds Engles sode Kvad , I Oret Sang , som gjor En glad , I Munden Honning underlig , I Hjertet Dugg af Himmerigl 17. Min Hu staar altid efter dig , O Jesu mm , kom vist til mig l Nåar vil du gjore mig saa glad ? Moet mig med dig , det er min Mad l 18. Her gaar jeg syg af stor Attråa , Kan jeg et venllgt <b>Oie</b> faa , Det drypper Honning i mit Saar , Ieg lives op , og styrket staar . 19. Jesu den Hjertens Skat , jeg veed , Jesu , du himmelsk Yndighed , Og Godhed ubegribelig , Din <b>Kjcerlighed</b> har tvunget mig ! 20. Dig , Jesu , har jeg valt mig ud , Ieg vcere vll din Hjertens Brud , Bort fra mig <b>selv</b> jeg gjerne dor , Og lever dig , som det sig bor . 21. O Jesu , op dit <b>Hjerte</b> luk , Og lyt til unne Lcengsels Suk , Saa inderlig med Graad paa Kind Ieg raaber : Jesu , Jesu min ! 22. Ja hvor jeg om i Verden gaar , Min Hu ul ene staar ; Hvor blev jeg glad , der jeq ham fanot , Som Dugg for Solen Sorgen svcmdt !

<h4>3835</h4>under sine Vinger . O naadige Gud , Et beder jeg dig om i denne Morgenstund , det noegte du mig ikte : lad mig faa vcere dit ljcere Varn , led mig idag med din trofaste Faderhaand og hold alt Ondt borte fra mig . Lad ingen uren Tanke besmitte min Sjcrl , ingen ond Lyst opstaa i mit <b>Hjerte</b> , ingen Synd faa Magt over mig . Lad mig vandre i <b>Kjcerlighed</b> med alle Mine og med alle Mennester , og med din gode Aand og dit hellige Ord lyse du for mig og lede mine Fodder paa Fredens Veil End beder jeg mere : Kjoere Fader , bliv aldrig saa vred , at du enten bortvender eller forvender dit salige Ord og dine hoivoerdige Salramenter , at de jo blive i dereS rette Vrug oS til Salighed indtil Verdens Ende ! O Gud , udss Naade over vor Konge og det kongelige Hus ; velsign Kongens Naad og alle dem , som gaa ham til Håande med at styre Land og Rige , skikke alt til Rette og sve Ret og Retfcerdighed . O Gud velsigne dem , som loere I lys <b>sclv</b> for dem , at de maa se og ftnde ret Vei og aldrig lede os vild fra <b>Sandheden</b> . O Herre , afvend Hunger og Dyrtid , Krig og Blodsiyrtning , Ilds og Vands Nod og farlige Sygdommel tilvend os derimod Lykke og Belsignelse baade timellg og aandelig l Gud give Fred i vore Dagel Gud velsigne vor Ager og Eng , vor Noering og Vjerging , og alt det , enhver i sit Embede tåger sig fore ! Herre glem aldrig den Fattige , forsvar den Fortrcrngte , hor Enter og Faderlsse ! Voer naadig imod alle , thi alle have din Naade behov , og for Alting bevar mig arme Synder i denne Dag og hver Dag med alt det , jeg eier og har , med ( Husbonde , <b>Hustru</b> , Foroeldre , Born ) Slcegt og Venner , Gods og Formue fra al velfortjent og ovcrhoengende Ulykte ! Gud bedre mig , mine Synder ere mange og med min Omvendelse og min Forbedring gaar det tungt og smaat , saa jeg ofte frvgter , at Hevnens Dag er kommen , og at du Me mere vil bcere over med mine Synder og have Langmodighed med mig . Men , at Herre , lad dit <b>Hjerte</b> dog ilte vendes faa aldeles fra mig ! voer endnu naadig for Jesu Styld , og lab mig ilte sege dig forgjceves . Lad din Aand tomme min Strebelighed til Hjcelp , vcelte mig hver Morgen til ny Lydighed . og gjennemtrange mig hver Dag med ny Kraft til Helliggjorelse . Ophold mig , Herre Jesu , i Naadens Samfund med dig ! Og nåar det lider til min sidste Stund , ba lad mig hore denne gladelige Rost : Kom , min Faders Velsignede , og arv det Rige , den Glcede og Salighed, som dig har vceret beredt for Verdens Begyndelsel Dig ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009091503033'target=_>Ahlfeld, Friedrich, 1866, Menneskelivet</a><h4>146</h4>os l « 3 s fra ham . Ja , om det ogsaa skeer , at vi en Stund fare vild , om ogsaa Livets <b>Sand</b> og Stsv lcegger sig over hine Scedekorn , saa vedbliver der dog at vcere Liv i dem , og i rette Tid skyde de op. Intet har oftere bragt forvildede og ulykkelige Syndere til Omvendelse end Tilbageblikket paa deres gudfrygtige og salige Ungdom , end Erindringen af de Stykker af det guddommelige Ord , som de havde lcert i deres Barndom . la , netop disse Ord have vceret en Stottestav for Mange i Dodens nwrke Dal . — Men ogsaa for Eder , I Forceldre , er dette Arbeide paa Eders Vorns Frelse til stor Velsignelse. Kun den Fader og den <b>Moder</b> , som have den himmelske Fader staaende bagved sig , som rsgte deres Gjerning i hans Navn og henvise til ham , staae i det rette , vcerdige Lys for Bsrnene . Der hviler en Glands af den guddommelige Majestcet om deres Hoveder , som fremkalder en langt hsiere Wrefrygt og en sandere Lydighed end al blodagtig <b>Kjcerlighed</b> og al streng Optugtelfe . Men dette er en Frugt , som mere viser sig i det Mre , en anden fremkommer i dit eget <b>Hjerte</b> . Guds Ord har en desto storre Magt , jo eenfoldigere det tages . Den , der jevnt og eenfoldigt fortceller sine Bsrn det , den , der nedlader sig til dem , ligesom Gud <b>selv</b> i Aabenbaringen har nedladt sig til os , vil ogsaa fsle dets Indtryk paa sit eget <b>Hjerte</b> , og det vil smage ham langt sodere , end om han studerede det med stor Lcerdom og udtalte sig om det i hsie Talemaader . Han bliver et Varn med sine Bmn , og hans egen indre Modtagelighed , hans Fred og Glcede blive unge igjen . Barndommens Sol skinner endnu engang i Dalen , og Blomsterne blomstre endnu engang ved Bcekken . — Derfor , I Fcrdre og <b>Modre</b> , forsommer ikke denne dyrebare Gjerning . Unge Planter maa man vande omhyggeligt . Saa vand da ogsaa trofast de guddommelige Planter i din Have , Du vånder derved tillige Livstrceet i Dig <b>selv</b> , der saa let visner i Livets Tsrke , og Du skaffer Dig <b>selv</b> den reneste Glcede , som denne Pilegrimstid har at frembyde . I Skotland lever en Lcege , der er meget bersmt som Videnskabsmand ; men fremfor Alt er han en stille , ydmyg Christen . Om sine Skrifter pleier han at sige , at de ikke ere hans , men Guds Vcerker ; det Gode , som er i dem , har han lcert af Gud . Denne Mand blev engang opfordret af en Fremmed til at besoge et andet Land , hvor hans Navn var velbekjendt , og hvor han vilde msde megen Anerkjendelse og blive meget ceret . Han svarede :  En Aften i Kredsen af mine Born er mig kjcerere end alle ZEresbeviisninger". Herre Jesus , giv os saadanne Fcedre . Herre Jesus , Du , som har våndet og frugtbargjort Verden med dit dyrebare Blod , vand vore Huse dermed , at der maa fsdes Dig Bsrn i dem forn Dugdraaber i Morgenrøden . Amen .

<h4>227</h4>Du samler Bsrnene , o Gud , under Foreldrenes <b>Kjerlighed</b> og Varetegt som i Skyggen af et grsnt og frugtbart Tre ; men Du omhugger ogsaa Treet , naar det behager Dig : Du gjsr Vornene foreldreløse , Du bersver dem de fsrste Venner , som de have paa Jorden . Herre , Du gjsr dette for at give dit Fadernavn Vidnesbyrd i Vernenes Liv . Du er jo den rette Fader for alle Bsrn i Himlen og paa Jorden , og Du kalder Dig jo <b>selv</b> Enkernes Forsorger og de Faderlsses Fader . O , Herre , saa hjelp da , at alle Foreldreløse maae ssge Dig med den rette Alvor og vandre paa deres Faders Veie . Vogt og bestjerm de forladte Vsrn , drag en hellig Muur omkring dem , at Fjenden , hvis Lyst det er at forderve Legeme og Sjel , holder sig borte fra dem . Lad det slaae ned med Lynets Kraft i Forfsrernes Hjerter , at disse stakkels Born i serlig Betydning ere dine Born , at den , der angriber dem , angriber din Slegt ; lad dem bceve forferdede tilbage ved denne Tanke . O , Herre , giv din christelige Menighed Naade og <b>Kjcerlighed</b> til at betragte saadanne stakkels Born som dens egne , til at trede i Faders og Moders Sted for dem , flittigt bede for dem og vaage over dem som over betroede Skatte . Ja , Herre , hjelp , at vore Foreldreløse maae vore op til LEre for Dig , der er deres Fader , deres Forlsser , deres Skjold og meget store Lsn . Amen . Naar man seer sig om efter Steder , hvor der i <b>Sandhed</b> hersker Jammer , saa dveler <b>Blikket</b> vistnok iscer ved de Huse , hvor Gud Herren har bortkaldt Foreldrene og efterladt en Skare smaa Born . Det er haardt , naar det fryser , at gaae ud i Kulden fra den varme Stue , det er haardt at drage bort til fremmede Lande fra det fedrene Huus ; men langt koldere er doa Overgangen fra trofaste Foreldres kjerlige Pleie til at vere uden Fader eller <b>Moder</b> . Man har sammenlignet de Foreldreløse med unge Treer i en Have , hvor Gjerdet er revet ned , med Viinranker , hvis Stetter Stormen har omkastet, med en Hytte , hvis Tag Uveiret har fort bort med sig . I den hellige Skrift , og i alle Sprog ligger der altid en ssrgelig Betydning i Ordene  Enker og Faderlose " . Den tidligere Tryghed og Sikkerhed er forsvunden ved Foreldrenes Dsd , en god Deel af Solskinnet er borte fra Ungdommens Have . Og <b>selv</b> om Gud kun tåger den ene af Foreldrene bort , saa foler Barnet dog allerede Savnet af denne tungt nok . Med Faderen er Husets Pille sonderbrudt, med Moderen er Sjelen i den stille huuslige Glede forsvunden , der er ingen rigtig Huuslighed mere . Og hvor sjeldent kan den ene

<h4>230</h4>tilbageblevne Deel med sine Gaver og sit serlige Kald kun nogenlunde udfylde den dobbelte Stilling ! Og naar nu begge ere borte ? Vi have oplevet det Tilfelde , at en <b>Moder</b> , som var Enke , dode om Natten , medens Barnene sov . Ingen Veninde stod ved hendes Seng og bad med hende og trykkede hendes Bine til . Om Morgenen stode Bsrnene op og klcedte sig stille paa . „ Lader os gaae sagte for ikke at forstyrre <b>Moder</b> , hun sover endnu " , sagde de til hverandre ; men da hun bestandig blev ved at sove , kom de endelig frem med det Spsrgsmaal: „ <b>Moder</b> , vil Du ikke give os Frokost idag , vi ere sultne ? Men Moderen svarer dem ikke mere . Da bankede de endelig paa Nabokonens Dsr og bad hende :  Kom dog ind til os , <b>Moder</b> vil slet ikke vaagne idag . " Og hun kom og saae , hvad der var skeet , og sagde til Vernene :  Kjere Vsrn , Eders <b>Moder</b> vaagner aldrig mere , vor Herre har kaldet hende til sig , kom ind med til mig . " I saadanne Aieblikke gaaer Smerten hen over Barnets <b>Hjerte</b> , som naar Hagelveiret pidsker de unge Blade as Treerne om Foraaret . — Men <b>selv</b> hvor Elendigheden ikke indfinder sig saa megtigt og uventet , have vi dog ofte feet en übeskrivelig Smerte male sig paa 3 e unge Ansigter , naar den sidste as Foreldrene blev baaren bort . O , hvad har det vel at betyde , naar Fuglenes Rede bliver forstyrret ! De sorge nogle Dage Paa den tomme Green , og saa bygge de sig atter en ny Rede . Men et Barn faaer aldrig igjen Foreldre . <b>Selv</b> om Gud skjenker det nok saa trofaste Pleieforeldre , saa faaer det dog aldrig den <b>Moder</b> igjen , der har baaret det under sit <b>Hjerte</b> . Med Foreldrene vare Vernene ester Guds Ordning sammenvoxede , de Andre maae de venne sig til . Og desuden forbliver den lille Skare sjelden samlet ; den Ene bliver anbragt her , den Anden hist hos forskjellige Pleieforeldre og barmhjertige Mennesker . Og hvor kummerligt , navnlig med Hensyn til <b>Kjerlighed</b> , bliver der ofte ssrget for dem ! — Dog fsrst komme vi til det SpSrgsmaal :  Hvorfor lader Gud dette skee ? " Kjere Leser , her maae vi strax svare :  Han holder ikke as saadanne Sporgsmaal , og han besvarer dem som oftest heller ikke i denne Timelighed . Ofte forstaae vi fsrst i de senere Aar nogle af hans underlige Forelser , ofte lader han os ogsaa vente til hiin store Dag , hvor der skal gives Svar paa alle Spsrgsmaal , eller hvor vi intet Svar mere begjere , fordi vi da skue klart , hvad der var hans Raadslutning . Dog ville vi i al Skrobelighed give et Svar . En gammel from Christen sammenligner Gud Herren med en klog Gartner , der afhugger de store Grene , som dekke over de unge Planter i Haven og betage dem Solskinnet , for at de knnne stue op til Solen , og for at den kan skinne umiddelbart ned paa dem . Saaledes borttager Gud Foreldrene , de sterke Grene , der brede sig ud over Bornene , for at disse kunne stue lige op til Naadeus Sol . De stulle tidligt fores hen til Herren , lere at troe , bede , at omgaaes med ham og forlade sig paa ham . " Der er noget <b>Sandt</b> i dette Svar . Ak , hvor ofte have ikke foreldreløse Vsrn lert at bede auderledes end fsr , da de ikke stode alene ; hvor

<h4>232</h4>har ikke Fadervor faaet en ganske anden Betydning for dem ! — Fremdeles tåger Gud ofte Foreldrene bort , for at de ikke skulle fordcerve Vernene ved deres Vantro og Synd . Rige og Yppige Folks Born ere ofte blevne foreldrelose , for at de ikke stulle inddrages i Mvighed og ugudelig Verdslighed ; derfor bragte Gud dem i et Huus , hvor de lcerte at arbeide , at bede og soge ham . Fattige Folks Born , som Intet kjendte til Frelsen og Frelsens Vei , bragte han ofte til et troende Huus eller til et Vaisenhuus , hvor en from Forstander gjorde for Bornene , hvad deres egne Forceldre ikke havde gjort , idet han forte dem til Herren ; og saaledes blev Vaisenhuset eller Pleieforeldrcnes Huus det nye Livs Vugge for dem . la , mange stakkels Born have i deres modnere Aar bekjendt :  Min Fader i Himlen , da jeg mistede min jordiske Fader , da fandt jeg Dig . " — Endelig lader Gud ogsaa mange Born miste deres Forceldre , for at <b>Kjerligheden</b> kan blive levende i Menigheden , for at Menigheden kan blive en <b>Moder</b> for disse Born . O , at den dog altid i <b>Sandhed</b> maatte blive dette , at den i den rette Samfundsaand maatte bekjende :  Det er vore Born , vi maae trccde i Forceldres Sted for dem ! " Men Gud naade os , hvorledes blive de Foreldrelose ofte behandlede ! Det er os bekjendt , at der i visse Menigheder er blevet holdt formelige Licitationer , hvor de stakkels Born tilsloges den mindst Bydende . Man har behandlet deres Opdragelse som en almindelig Communesag , som en Vei , der skulde udbedres , som et Dige , der skulde ovfores . Man har ikke spurgt , hvem Bornene kom til , hvorledes de bleve ovdragne , hvortil de bleve brugte . Hvormange Foreldrelose have ikke maattet gaae om fra Dor til Dor og tigge for at ovvekke en Medlidenhed , der kunde indbringe dem <b>selv</b> og deres Pleieforeldre Noget ! Ofte har Menigheden viist sig som en ugudelig Stifmoder mod de stakkels Born — og deres Formyndere have hele Aar igjennem ikke erkyndiget sig om dem . Naar Foreldrene have lukket deres Bine , bor de ncermeste Slcegtninge eller andre Venner , Fadderne og Formynderne opfylde deres Pligter . Hvortil er ellers Slegtfkabet ? Naar Moderen er dod fra et Lam i Hjorden , saa give de andre Faar det <b>Die</b> ; og nesten hos alle Folkeslag findes Sagn om , at en Hind , ja endogsaa en Ulvinde har givet <b>Die</b> til udsatte Born og er bleven deres Pleiemoder. Ere I da ikke meget Mere end de ? Skal der ikke boe en ganske anden Slags <b>Kjerlighed</b> i Guds Born end i de stakkels ufornuftige Skabninger ? — I forsømmelige og ukjerlige Forstandere og Slegtninge , og I Mammonstjenere , som udstrekke Eders Hender ester de Foreldreloses Eiendom , ville I ikke lade Eder afholde fra Synden ved <b>Kjerlighed</b> til Eders Frelser og de staktels Born , saa forferdes dog for den almegtige Guds Vrede . Han bevarer og cpreiser de Faderlose . Og hvad han bevarer , det foragte og forsomme og forderve I ! Naar man gjennemleser Guds Ord , - vil man finde , at han iser tåger Enker og Faderlose i Beskyttelse . Han har med sine Bud og sine Trusler draget faste Mure omkring dem . Med Magt vil han forstyrre de Ugudeliges Vei , at de ikke gjore dem

<h4>254</h4>kan der ikke udvikle sig nogen Ro og Dybde i Vesenet og Charakteren; thi hans Sjel beveger sig paa Overfladen , og han er afhengig af Menneskenes Dom . Han vil heller aldrig naae til nogen <b>sand</b> Eenhed og Fasthed , thi han har ti eller tyve Herrer , som han maa rette sig ester . Ethvert Menneskeansigt , Enhver , der kan selde en Dom , er ham en Gud , og da de forskjellige Menneskers Dom er forskjellig , har han intet fast Punkt at staae paa , men svcever altid i Luften . Med Rette kalder Paulus den LEre , forn man soger hos Mennnefker , en forfengelig 3 Ere . Den er uden Kjerne , og den , der jager efter den , faaer kun en Skal uden Indhold . <b>Selv</b> det Helligste maa tjene denne Afgud . A . H . Francke gik engang om Sommeren , med en af sine Venner over en Mark . Da hsrte de to Bsrn , som sade inde i Kornet , bede med hinanden , og i stille Andagt og Glede lyttede de til . Men da det ene af Vsrnene var ferdigt med at bede , sagde det til det andet :  Hsrer Du , hvor smukt jeg kan bede ? " og ved disse Ord var det , som om de to Venner fik en Spand koldt Vand over Hovedet . Forfcengelighedens Orm havde ogsaa gnavet fig ind i Bonnen ; thi den skaaner Intet . Det Menneske , i hvis <b>Hjerte</b> den har bidt sig fast , lerer aldrig at kjende den sande Fred og Ro . Hans Liv er feberagtigt ; hans Puls slaaer hurtigt , hvergang hans Tsrst efter 2 Ere kan blive tilfredsstillet ; og naar der ingen Leilighed er dertil , eller naar han maae staae tilbage for Andre , bliver han kold og fsleslos . Ak , hvor fattigt bliver dog et saadant Menneske! Hvergang han forbigaaes med Taushed , hvergang han dadles , er det , som en kold Skylregn styrtede ned over ham . Og hvor meget der end iskjenkes ham af 3 Erens berusende Drik , bliver hans Torst dog aldrig stillet ; thi A3re er som Havvand , jo mere man. drikker deraf , desto mere torster man. Hoffcerdighed er som en Ulv , der aldrig bliver mcet . <b>Sand</b> Ro og Tilfredshed er kun at finde i Gud . — Dette , kjere Lesere , er den ene Side af den Ulykke , som den forfengelige 3 Ere styrter os i . Vi blive indvortes fattige ved denne Nigdom . — Men der gives ogsaa en anden . LErgjerrigheden er <b>Moder</b> til mange Synder . Hovmod var , efter vore gamle Theologers Mening , Aarsag til Djevelens Fald , og ved at indbilde vore fSrste Foreldre , at de vilde blive ligesom Gud , lokkede han dem til at synde . Sirach kalder ( 10 , 14 ) Hofferdighet ) en Vegyndelse til Synd ; Tobias siger ( 4 , 17 ) :  Udi Hovmod er Fordervelse og megen Übestandighed " , og vor Tert setter Trods og Avind i Forbindelse med forfengelig A3re . De ere ogsaa nogle af de fsrste Folger , som den har . Den , der gjor LEren til sin Gud , vil tilegne sig <b>selv</b> al Anerkjendelse og Noes . Han bliver kold mod sine Medmennesker , og de blive ham tilsidst kun Trin paa den Stige , som han vil heve sig op paa . Han bliver hjertelss , han har ingen <b>Kjerlighed</b> til Andre , og han <b>moder</b> heller ingen <b>Kjerlighed</b> ; tilsidst har han ingen Venner mere . Sirach siger ( 10 , 7 ) :  Hovmod er hadelig for Herren og Menneskene " . — Den Hofferdige bliver skarp og nretferdig i sin Dom over Andre for <b>selv</b> at kunne indtage det overste Trin . Pharisceeren,

<h4>625</h4>tyende Ssstre udfyldte hinanden , hvorledes de modtoge af hinanden og igjen meddeelte . Havde de levet adskilte , kunde enhver af dem vcere kommen i mange og store , sjelelige Farer . — I den nyere Tid er der ved Guds Naade ogsaa paa flere Steder i den evangeliske Kirke blevet aabnet en stsrre Arbeidsmark for de ugifte <b>Kvinder</b> , idet de uddanne sig i Diakonisseanstalter til at pleie Syge og gaae omkring i Hospitalerne eller i Familierne for udfsre denne Gjerning i Herrens Tjeneste og med Herrens Ord . En saadan tjenende <b>Kjcerlighed</b> opfylder Sjcelen og fremmer dens Vext til ham , som er Hovedet , til Jesus Christus ( Eph . 4 , 15 ) . For Bieblikket befinder en heel Skare af disse evangeliske Sostre sig i Syrien , besijeftigede med at lege de Såar , som muhamedanfk Fjendskab har slaaet Herrens Tilhengere i Libanon og Damaskus . Ligeledes arbeide ugifte Kviuder i Pleiestiftelser og Asyler eller lignende Anstalter ; ogsaa i Stolerne have de nu sundet en langt stsrre Virkekreds end fsr . I alle store Steder bestaaer der Foreninger for at redde fattige Familier , faldne <b>Kvinder</b> , forssmte Born og ovdrage ucegte eller foreldrelsse Born . Paa disse Maader kunne <b>Kvinderne</b> tjene Herren ved at tåge sig af deres stakkels Sostre , og vaa samme Tid modarbeide de Dsden , Egenkjerlighedcn og Forstenelsen i deres eget <b>Hjerte</b> . Hvor rigt i Herren var ikke den for nogen Tid siden i Hamborg afdode Amalie Sievekings Liv ! I hvormange Familier har , hun ikke vceklet Troen og afhjulpet den ydre Nod ! — — Dog bliver Hjemmet altid det egenlige Sted , hvor <b>Kvinden</b> skal leve og virke ; og den Pige , som Gud ikke skjenker noget eget Hjem , er i denne Henseende henviist til sine ncermeste Slcegtninge . De ugifte Dsttre hore hjemme i deres gifte Brsdres eller Sostres Huse . Ja , <b>selv</b> om de eie saa Meget , at de godt kunne leve for sig <b>selv</b> , saa bor de dog foretrekke en Plads i deres Vroders eller Sosters Huus . Vel <b>sandt</b> , Livet der kan blive besverligere , det kan ogsaa vcere forbundet med smaa Fortredeligheder; saa bliver det ogsaa til Gjengjeld langt mere velsignet og rigt . De staae Sosteren eller Svigerinden bi , de hjelpe med til at opdrage Bsrnene ; de indeslutte Huset langt inderligere i deres Bonner , naar de hore hjemme i det , end naar de staae udenfor det og betragte det i Afstand . De kunne , medens den gifte Soster eller Svigerinden besorger Huset , blive Voruenes fsrste Lererinder , eller omvendt ved at hjelpe med i Huusholouingen skaffe Sosteren Tid til at passe Bsrnene . De leve og bede og sorge og glede sig med Familien ; de hore til den , de vore sammen med den . De have deres Deel af Moderforgerne , men tilvisse ogsaa af Moderglederne . — > Fornemmelig horer den ugifte Soster hjemme i en Vroders Huus , som er bleven Enkemand . Der skal hun overtage en Moders hele Gjerning . Ligeledes trenger en Ssster , som er bleven Enke , til hendes Hjelp . Naar To staae sammen , kunne de bedre bere Korset ; det har jo to Arme , ved hver as dem stal En tåge fat . Gid dog ingen <b>Kvinde</b> vilde foretrekke sin Magelighed og Ro for en saadan sosterlig Bistand ; hun undgaaer alligevel ikte Korset ved at flygte for

<h4>637</h4>utydelige Ord ved bestandig at foresige det Noget . Hun har megen Taalmodighed med Barnet . Gaaer hendes Haab ikke i Opfyldelse i det forste Aar , saa fortsetter hun ufortrødent sine Anstrengelser i det andet . Men hvilken Smerte er det ikke , naar Lcegen endelig siger hende , eller naar hun <b>selv</b> maa sige sig :  Dit Varn er dovstnmt ! " Hvilken Smerte , naar hun mcerter , at hendes Barn ikke kan trcede paa sine Fodder , ikke kan staae eller gaae ! eller naar det har mistet Vinenes Lys i en eller anden svcer Sygdom ! Hvor vanskeligt sinder hun sig i den bittre <b>Sandhed</b> , at hun vel seer Barnet , men at det ikke kan see hende . Og endnu tungere er det , naar hun Aar ester Aar har ventet og arbeidet paa , at der skulde vise sig Åttringer af Forstand og Tcenkning hos Barnet , men der ingen kommer , og Barnet bestandig betragter hende med slsve » <b>Blik</b> . Og dog skulle Forceldre eller Slcrgtninge hverken fortvivle eller knurre derover . Gud har <b>selv</b> viist os , hvorledes disse stakkels Vcesener skulle behandles. I Modrenes Hjerter har han nedlagt en Drift , som med en ganske sceregen Kraft drager dem hen til det svage , syge og vanskabte Varn ; ja , han har nedlagt denne Drift i hele Folkestammer . I Alpedalene i Savoien findes der mange saakaldte Cretinere , det vil sige Mennesker , som ere Kroblinger baadr Paa Legeme og Sjcel . Ethvert Huus , hvori en Saadan horer hjemme , anseer ham for en Guds Gave , ja , for en Velsignelse , og <b>selv</b> de fattigste Folk vilde gjore sig en Samvittighed af at forssmme eller mishandle en saadan Syg . — Og saaledes skal enhver Familie betragte og elske et sygt Medlem af den som en Guds Gave . Om ogsaa Menneskenes Synd har bidraget Sit til Elendigheden , saa har dog Gud tilstedt og beskikket den ester sin übegribelige Raadslutning . Om ogsaa den guddommelige Styrelse her har ladet sin straffende Vrede tomme tilsyne , saa er det dog Guds Gjerning ; den Syge er dog , saaledes som han er , en Guds Gave . Derfor bsr den christelige <b>Kjcerlighed</b> netop i dette Forhold vise sig i hele sin Styrke og Trofasthed . Her tjener den ikke for Lon , her kan den i mange Tilfcelde aldrig blive gjengjeldt med en blot nogenlnnde tilsvarende <b>Kjcerlighed</b> . Her hedder det med fuld <b>Sandhed</b> :  For Herrens Skyld ! " O , hvor har dog denne Herre antaget sig de stakkels Kroblinger , da han vandrede paa Jorden ! Ofte saae det ud omkring ham som i et stort Lazareth . O , beder ham dog om at stjcenke Eder det samme kjcerlige <b>Hjerte</b> ! <b>Sand</b> , oprigtig <b>Kjcerlighed</b> er netop det christelige Liv og et Stykke af det evige Liv . Naar Tro og Haab have faaet deres Oftfyldelse og Fuldendelse , saa bliver <b>Kjcerligheden</b> endnu ( 1 Cor . 13 ) . Holder kun rigtig fast ved dette : Gud har givet os vore Syge til en øvelse i <b>Kjcerlighed</b> . Paa dem stulle vi lcere den storste Kunst i Verden . De stulle , mere end de snnde Medlemmer af vor Familie , tjene til at fremme vor Vcext i Herren . Jeg kjender Familier , hvem Herren har paalagt et saadant ' Kors i 10 , ja 20 Aar . Jeg kjender ogsaa Familier , som have forstaaet , at det var hans hellige Raadslntning . O , hvor have de ikke lcert at bede ! Hvor have de ikke lcert at elske

<h4>698</h4>min Lenke ; men der han var i Rom , sogte han saare flittelig efter mig og fandt mig . Herren give , at han maa finde Varmbjertighed hos Herren paa hiin Dag " ( 2 Tim . 1 , 16 — 18 ) . — Dog hvis man vilde udskrive alle de Steder i Guds Ord , hvor Barmhjertigheden omtales , maatte man udskrive hele den dyrebare Bog . Ikke <b>sandt</b> , unge Mgtefolk , ligesom Herrens Varmhjertighed er opgaaet over Eders Huus , saa stal ogsaa Eders Barmhjertighet ? lyse ud af det til Eders stakkels Medbrodre ! Men til hvem skal de » strekke sig ? — Vi behewe ikke at sporge lcenge . Herren siger : „ Fattige have I altid hos Eder . " Nu gives der vistnok Fattige , hvem ogsaa den hellige Skrift frakjender Ret til at nyde godt af Varmhjertigheden,,thi det hedder : „ Dersom Nogen ikke vil arbeide , bor han heller ikke have Fsde " ( 2 Thess . 3 , 10 ) . Hungeren stal drive dem til at gaae i sig <b>selv</b> og tåge sig noget Nyttigt for . la , naar vi med vore Gaver understøtte Synden , saa er endog Gavmildheden <b>selv</b> en Synd . Men derimod ere fattige Familier , som alvorligt strebe efter at tjene Herren og at cede deres Brod i deres Ansigts Sved , Syge , Gamle , Enker og Faderlose det Alter , paa hvilket Varmhjertighedens Offre stulle nedlegges . Der gives Familier , syge og elendige Mennesker , som trenge til bestandig at understøttes . Men undertiden opstaaer der ogsaa sccregne Tilfelde . Ofte staaer den bittreste Nod ved det nyfodte Barns Vugge , og da stal Forbarmelsen stille sig ved Siden heraf , for at den nye Himmelborgers fsrste <b>Blik</b> ikke stal speile sig i Foreldrenes Taarer . Naar en Families Forssrger doer , stal den christne Menighed give Vidnesbyrd om , at Herren endnu lever , og at den <b>selv</b> lever og tjener ham . Ofte mangle fattige Born ved deres Confirmation en ny Kledning , hvori de knnde mindes den Retferdighedens Kjortel og det Frelsens Kledebon , som Herren har ifsrt dem i Daaben ; da stal den christelige <b>Kjerlighed</b> oplade sin Haand , for at Erindringen herom kan vere levende <b>selv</b> hos den Fattigste . Paa Julefesten bor intet Huus , hvori man giver Herren LEren , vcere morkt , og Bordet i det ganske tomt . — Som i de Fattiges Liv , saaledes gives der , ogsaa i dit Liv sceregne Anledninger , som opfordre til Velgjorenhed . Paa alle Familiefester, paa dine Boms Daabsdage , paa Hostfesten , ved Enden af et lykkcligt Aar stal den Fattige glede sig med Dig . Paa den Dag , da Du gaacr til Alters , da Herren skjenker Dig saa Meget , skal Du ikke glemme den Trengende . Jeg har Skrifteborn , som med stille Trostab folge denne gamle Skik . Der er intet Sted , hvor Varmhjertigheden ikke kan finde Leilighed til at tjene Herren ved at komme hans Fattige til Hjelp . — Dog er det ikke gjort med blot at give ; der stal ogsaa gives med Glede . Gud elsker en glad Giver ( 2 Cor . 9 , 7 ) . Man stal lade sit <b>Hjerte</b> folge med Gaven . Gleder Eder med de Glade og greder med de Gredende ( Rom . 12 , 15 ) . Eders Von stal ogsaa ledsage Gaven , at den maa blive til Velsignelse for det indre og ydre Menneske . En Gave og en Hjelp uden <b>Kjerlighed</b> og Bon er en smuk , men indvendig huul Frugt .

<h4>700</h4>Det stille , venlige Arbeide paa Sjelene bsr ogsaa gaae Haaud i Haand hermed ; netop naar en Velgjerning modtages , ere Hjerterne allermeest modtagelige for den himmelske Velgjerning . Men i Stilhed bsr det Altsammen skee . Alle Phariseere , som lade blcese i Basuner for sig eller <b>selv</b> gjsre det , have deres Lon borte ( Match . 6 , 2 ) . Den venstre Haand stal ikke vide , hvad den hsire gjsr . Vi skulle heller ikke give til Andre for at fortjene Tak derved , saa at vi blive forstemte og fortrydelige , naar Takken udebliver . Ak , hvor ofte har Herren ikke hostet Utak as os , og dog giver han os altid nye Gaver !  Skik dit Bred til Sses , Du vil fiude det mange Dage ester , " skriver Salomon i Predikerens Bog ( Cap . 11 , 1 ) . Han mener :  Udsaa trostigt din Barmhjertighed , hvor Du slet intet Haab har om Gjengjceld , og Herren vil netop da gjengjelde Dig . " Vi stulle heller ikke vente , til Armoden banker paa vor Dor og vort <b>Hjerte</b> med sin Von . Den barmhjertige Gud er kommen vor Nod imode ; han har ogsaa stjcenket dem sin kjcere Son , som ikke kjendte noget til Forjettelsen om ham , som aldrig havde bedet om hans Komme . — Men hvor hente vi Kraft , <b>Kjerlighed</b> og Trostab til denne udholdende Samaritanertjeneste ? Ikke hos os <b>selv</b> , heller ikte hos de Fattige , men hos Herren . Tcenk kun ret flittigt paa , at Du skylder hans Barmhjertighed Alt , hvad Du har og er . Tcenk bestandig paa , at Du er Guds Barn , at Du er Christi Efterfolger og et Lem paa hans Legeme . Man maa i din Vandel kunne gjenkjende din Herres Billede , altsaa ogfaa i Barmhjeitigheden . Og dernest stal Du r enhver fattig og ulykkelig Christen see din Herre <b>selv</b> . Den , der bespiser den Hungrige , giver den Torstige at drikke , kleder den Nogne , besoger den Fangne og den Syge , tåger den Fremmede til sig , har gjort alt dette mod ham ( Match . 25 ) . De gamle Fedre i vor Kirke fortelle undertiden i deres Predikener paa deres barnlige Maade et Sagn , som stiller denne <b>Sandhed</b> i det klareste Lys . Det lyder saaledes : Der var engang to Vrodre , en fattig og en rig . Den Fattige havde en fed Hone og vilde byde sin Nabo til Gjest pace den . Hans <b>Hustru</b> var enig med ham heri , og formanede ham kun til ogsaa at indbyde den Herre Christus . Manden syntes godt herom , gik ind i Kirken , traadte hen for Crucifixet og bad Herren at komme til ham og tåge til Takke med Netterne . Da boier den Korsfestede Hovedet for dermed at antyde , at han vil komme . Manden siger det til sin <b>Kone</b> , og hun anretter Maaltidet . Kort for Spisetiden kommer en Tigger og beder om en Almisse . <b>Konen</b> siger :  Vi have rigtignok ikke mere end vor Hone ; men den statkels Mand stal dog , faae den ene Vinge . " Da <b>Konen</b> havde rettet an , gik Manden hen for Crucifiret og bad sin kjere Gjest indstendigt om dog at komme , Maden stod allerede paa Bordet . Da siger den Herre Christus :  leg har jo allerede veret hos Dig og spiist med Dig ! " Manden svarer :  leg har ikke seet Dig , Herre ! "  Io Du har , " siger Christus ,  jeg var den Tigger , der kom til din Dor for at bede om en Almisse, og hvem din <b>Kone</b> gav et Stykke af Honen . Gaa derfor kun

<h4>721</h4>og uftaalidelig Stav ; Du er henviist til Herren din Gud med hele dit Haab . Jesus Christus bliver langt snarere din eneste Skat , fordi ingen jordisk Skat strider med ham om dit <b>Hjerte</b> . Du bliver mere fortrolig med den inderlige og vedholdende Bon end den Rige , fordi Du alene scetter din Lid til Herren din Gud . Du har ingen anden Vrond at ose af end hans Rigdom og Varmhjertighed . Du er scerlig henviist til at bede med David ( Ps . 27 ) :  Til Dig har mit <b>Hjerte</b> sagt : ester de Ord :  soger mit Ansigt " , vil jeg soge dit Ansigt , o Herre ! Skjul ikke dit Ansigt for mig , forskyd ei i Vrede din Tjener ; Du har vceret min Hjcelv ; forstod mig ei og forlad mig ei , min Frelses Gud ! Thi min Fader og min <b>Moder</b> have forladt mig ; men Herren vil annamme mig . " — Du er scerlig henviist til Guds dyrebare Ord . Dig har intet Menneske indsat i sit Testamente , desto mere maa Du kvcege Dig ved Herrens Testamenter og Arvelod . — For Dig har Sondagen iscerdeleshed Vcerd og Betydning . For den Rige , der maaskee ikke har havt noget synderligt Arbeide i Ugens Lob , er den som en Brond med lunkent Vand ; men for Dig , som i Ordets egentlige Betydning ceder dit Brod i dit Ansigts Sved , er den som et frisk Kildevceld i ørkenen . — Din Sjcel vil lettere kunne fole den rette Hjemvee efter Herren end mangen rig Mands Sjcel . Du er kun lidet knyttet til Jorden ; den har ikke viist sig saa smuk for Dig , at Du derfor kunde onske din Dodstime opsat fra een Termin til en anden . Men Du maa i al din Moisommelighed heller ikke glemme , at det at vcere kjed af det jordiske Liv endnu ikke er den rette Hjemvee . Kun det Menneske foler den sande Hjemvee , som gjerne vil vcere med Christus og derfor har Lyst til at vandre herfra ( Phil . 1 , 23 ) . Herren har <b>selv</b> sagt til Dig , at Du er Himmeriges Rige ncermere end de , som have meget Gods . Det er letttere , at en Kameel gaacr igjennem et Naaleoie, end at en Rig kommer ind i Guds Rige ( Match . 19 , 24 ) . Den rige yngling s ^ aer for Naadens Dor , men han gaacr forbi , fordi Indgangen er ham for snever og lav . De Fattige gaae derimod ind og annamme Rigdomme af Herren . Istedetfor at knurre over din Fattigdom , skal Du hellere takke Herren , fordi han har bygget din Hytte saa ncer ved Frelsens Port , og fordi hans Indbydelser lyde saa klart og tydeligt til Dig . Hold kun fast ved din Frelser . Forsaml Dig med Dine omkring det kjcere Guds Ord . O , det skaffer dyrebare og uforglemmelige Aftener i den fattige Hytte , naar Faderen lceser i <b>Sandhedens</b> evige Ord , og Hustruen og Bornene lytte dertil ! Da bessger den Hellig Aand den lille Menighed , Hjerterne gjennemstrommes af salige Folelser , den dunkle Lampe brcender klart , Fattigdommen glemmes , I ere glade i Herren , Guds <b>Kjcerlighed</b> udgydes over Eder ; I fole og ane , at I eie en übeskrivelig rig Arv . . — Og lad Dig derhos for ingen Priis berove de tjcere Sondage . Du veed , at de ret egentlig ere Herrens Naadedage. Det evige Livs Morgenrode skinner da i din Hytte , Du har en Forsmag paa den store Sabbath , paa den store Hvile hos Gud .

<h4>860</h4>blive ofte ved at vcere fremmede i Huset trods al <b>Kjcerlighed</b> og Troskab . Vel opgive sEgtefolk , som endnu ere i deres kraftige Alder , nodigt det Haab , at Gud vil velsigne dem med Born , og ovfcette derfor et saadant Skridt fra det ene Aar til det andet . Og Gud kan vistnok endnu ovfylde deres Haab ; men han sver iscer Barmhjertighed mod dem , forn <b>selv</b> have ovet den . Og hvad der maafkee afkortes i de andre Boms Arvedeel for det fremmede Barns Skyld , det kan den rige Gud erstatte tusindfold . Jeg Horte for ikke lcenge siden en barnlos , gammel <b>Kone</b> beklage sig saaledes :  leg og min falig Mand have forsomt Noget . Vi skulde allerede for 25 Aar siden have taget et fattigt , forceldrelost Barn til os . Men da vi indfaae vor Feil , var det for sildigt . Vi vare for gamle til at oftdrage et lille Barn og havde vi taget et sterre til os , saa vilde den Smule Arv , vi efterlade , have faaet en altfor stor Betydning for det ; vor Formue havde da maaskee ligget det mere daa Hjertet end vi <b>selv</b> . Jeg foler godt , at Alle , som i de senere Aar have villet slutte sig til os , havde Bihensigter ; men jeg foler ogsaa dybt , hvor eensom jeg staaer i min ncermeste Kreds . " — Man kan her i <b>Sandhed</b> sige :  Hvad Du gjor , det gjor svart ! " Og da lade Naadens Gud Dig ogsaa faae Glcede af et saadant Barn , som om det var dit eget , Du havde oftdraget i Herrens Frygt . Men os Alle , der <b>selv</b> have Born , bevare han for den forfcerdelige Jammer at maatte udraabe paa hiin store Dag : „ Salige ere de Ufrugtsommelige og de Liv , som ikke fodte , og de Bryster , som ikke gave <b>Die</b> " ( Lue . 23 , 29 ) . la , kjcere Fader i Hunlen , for Jesu Christi Skyld . Amen .

<h4>1006</h4>Vor Herre Jesus Christus , bank endnu engang paa Doren med al din Magt og <b>Kjcerlighed</b> , forend Legeme og Sjcel skilles ad . Viis os endnn engang i den Hellig Aand al vor Skyld , men ogsaa al din <b>Kjcerlighed</b> og Naade . Hjcelp , at vi endnu engang maae fceste <b>Blikket</b> klart og tydeligt paa os <b>selv</b> og paa Dig . Vcek i os Alle , fsr vor sidste Time kommer , en inderlig Hunger og Torst ester dit Legeme og Blod . Opluk <b>selv</b> vore Hjerter , at de maae staae aabne for Dig . Lad os endnu engang aflcegge en <b>sand</b> og dyb Syndsbekjendelse , hvori vi for dit Aasyn holde Dom over hele vort Liv fra Barndommen af . Men skjcenk os da ogsaa en fast Tro , at vi uden Tvivl og Vakten kaste al vor Ungdoms og Alderdoms Synd paa Dig , som er bleven os Viisdom fra Gud og Retfcerdighed og Helliggjsrelse og Forløsning ( 1 Cor . 1 , 30 ) . Og troed saa ind hos os , kjcere , trofaste Frelser , og hold Nadvere med os . Giv os Vidnesbyrd i den Hellig Nand , at Du har taget al vor Skyld fra os , at vi ere blevne Eet med Dig . Lad Guds Fred drage ind i det stakkels bange <b>Hjerte</b> , og giv os Styrke og Tillid

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010012603001'target=_>Wexels, W.A., 1837, Forklaring over Det nye Testamentes Skrifter</a><h4>3106</h4>de under Syndens og Dodens Magt Fangne , der ved Christi Ord , i Kraft af hans Forsonings - Dod , stulde , saasandt de med troende <b>Hjerte</b> annammede hans Ord , udfries af Syndens og af Dodens Rige . De aandelig Blindes <b>Oie</b> stulde oplades for Guds Herlighed , og de under Lovens Aag Bundne og Plagede stulde , retferdiggjorte af Naade , under Christi lette og gavnlige Aag nyde Guds Boms herlige Frihed . Sml . Matth . 11,29 .30 . ( og Forkl . dertil ) . Joh . 8 , 36. Gal . 3 , 13. 23 - 26. ; 4 , 1 - 7. Istedetfor de to sidste Led har Propheten blot : " og ( at udraabe ) for de Bundne Opladelse (LosninD, Befrielse ) . " Evangelisten udtrykker det Begreb af Forlosning, som ligger deri , baade med de Ord , han fandt i den gamle greske Oversettelse , som dengang almindeligen brugtes af de gresttalende loder og hvori det hedder : " ( at forkynde ) de Blinde Synet , " de i Morket Werende Lys , og med dem , der svevede ham for Tanke som Prophetens egne , hvorved han dog maastee ogsaa havde Es . 68 , 6. for <b>Oie</b> ( " at udlade de Fortrykte frie " ) . — " HErrens behagelige Aar " er het samme som Naades- Tiden fra HErren , da han vilde lade sit Velbehag , sin Velsignelse hvile over Mennestene formedelst sin elskelige Ssn . Denne Naades-Tid var vistnok navnlig "Mennestens Sons Dage " ( 19 , 42. , 44. ) eller den Tid , da han vandrede i Kjodet paa Jorden og aabenbarede sin Herlighed fuld af Naade og <b>Sandhed</b> , men den udstrekker sig med denne Naade og <b>Sandhed</b> , med hans Gjerningers Frugt , gjennem den hele Tid fra hans Komme og til Dommens Dag . See Lue . 1 , 68 ff . ; 2 , 14. Ap . Gjem . 3 , 19. 2 Cor . 6 , 2. — Skjondt nu vistnok Talen paa hele dette Sted , som Jesus foreleste i Synagogen , nermest er om Christi prophetiske Embede , saa maa hans Guddoms-Magt og <b>Kjcerlighed</b> og hans Midler- Embede dog derfra ei udelukkes , og vi maae legge Merke til , at han forkynder , hvad han <b>selv</b> stulde give Mennestene , hvad han stulde erhverve dem ved sit Gjenlosnings-Verk og meddele dem ved sin Aand gjennem Ordet . Sml . iovrigt med nerverende Prophetie Es . 42 , 1 - 4. ( Matth . 12 , 18 - 21. ) .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092403020'target=_>Ahlfeld, Friedrich, 1865, Menneskelivet i Guds Ords Lys</a><h4>172</h4>mnn vente , indtil et fremmed <b>Hjerte</b> , forn har mere <b>Kjerlighed</b> end Moderhjertet , kommer og tager det til sig . Ve en faadcm <b>Moder</b> ! Hun betragter ikke sit Varn som nogen Guds Gave , men som en Byrde , der er hende paalagt mod hendes Villie , som en Anklager mod hendes Synd og Skyld . Huu veed intet af det Ord : „ Men nåar hun har fodt Barnet , kommer hun ikte mere den Trengsel ihu , nf Glede over nt et Menneske er fodt til Verden . " Trengselen lever med Barnet , ng derfor vil hun vere fri for det . — Dog , vi ville ikte tale mere om disse <b>Modre</b> uden Moderhjerte . Men ner til dem grendse de <b>Modre</b> i de store Byer , som efter Fonselen sette sit Barn i en af de Pleiestiftelfer for fattige Dicborn , forn gjerne der findes . Naar faadnnne Born begynde nt face lidt Forstand , <b>moder</b> hellerikke deres <b>Oie</b> nogen <b>Moder</b> , men en fremmed <b>Kvinde</b> , forn gjor Smnnborns Opfodelse — Opdragelse kunne vi ikke kalde det — til et Hacmdverk , et Levebrod . Moderen hnr frasagt sig Moicn med sit Barn , men dermed ogsaa Gleden deraf . Ikke stort bedre er det i de fleste Tilfelder , nåar Born , hvad man kalder , udfettes til Opfostring . Om end dette Skridt ofte knn vere en forgelig Nodvendighed , faa er denne dog fordetmeste en Frugt af Synden . Snndnnne <b>Modre</b> stulde kim ret betenke , hvorledes Synden forgifter nl Glede , hvorledes den losner felv de inderligste Forholde , saaledes som Barnets til Moderen . Sjelden , sjelden betragter Barnet senere Moderen som sin rette <b>Moder</b> . Hun har jo hellerikke <b>selv</b> betragtet Barnet rigtigt som sit Barn , har ikle med snur Moie og vaagne Netter kjobt sig en Plads i Barnets <b>Hjerte</b> . Gud havde givet hende det som en Gave , og hun har ikke villet have det som saadan . — Men <b>selv</b> bortseet fra disse Afveie , staaer ofte Sorgen i Gledens Sted ved det stakkets Barns Vugge . Istedetfor Tnknemmeligheds- og Gledesstraaler feer Barnet Knmmerens Taarer i Foreldrenes <b>Oine</b> . Istedetfor den forste Vuggesang horer det Sukke . Ofte teuke fattige Foreldre : „ Varneflokten bliver for stor . Hvem stal sode den ? Vi kunde med Nod og Neppe staffe Brod til dem , vi havde , og nu stjeuker Gud os eet til ! " Mangen Fader har allerede knurret under siv Hustrues Svangerstab og siden seet suurt til sit yngste Barns Vugge . Og dog er dette ilke Andet end Mangel paa Tro , Mangel paa Mod . Dn er ikke Barnets overste Fader . Dets Skjebne styres af Ham , fra hvem alt Fadernavn er i Himlene og pnn Jorden . Hau hnr cmtaget det til sit Barn i den hellige Daab ; Han har ladet det dobe til sit Navn ; Han har ikte fagt : „ Flokk.n bliver for stor ; jeg vil ikke mere lade noget Barn dobes til mit Navn , min Naade og Omsorg . " Nn

<h4>233</h4>mere stulde kunnet losrive sig fra den Forjettede . — Og saaledes vil det ogsaa gaae med dine Born . I den tidlige Barndom er Hjertet bloot og Hukommelsen ung . Hvad man da lerer , det indpreger sig dybt og glemmes fordetmeste aldrig igjen . — Men ikke nok hermed . Hvad vi lere i vore forste Barneaar , og navnlig af vor <b>Moder</b> , det seer anderledes ud end hvad vi senere lere . Morgenduggen henger ved det ; Modertjerligheden med den hele Erindring om disse lykkelige , sorglose Aar omsvever det . Naar vi fenere treffe snndnnne Udsngn og Fortellinger igjen , tale de et gcmste andet Sprog end de , vi siden have lert . Dette ligger ikke i Guds Ord , som om een Deel as samme var kraftigere end en anden . Det ligger i Tiden og Hjertets Modtagelighet » . Disse forste Udsagn og Fortellinger have stacet Rodder i hele Livet . Saaledes er det ogsaa med Seden ude paa Ågeren . Hvad der saces itide om Vaaren , fester dybest Rod . Om ogsaa en anden Ager senere besaces med samme Sed , modstacer him dog bedre Torke og lonner Lcmdmanden med en rigere Afgrode . Hvad der fra forst af er indpreger i Barnets <b>Hjerte</b> , det giver dets Liv sin Retning . Historien kjender en Mand , som allerede i sine tidligste Aar legede med en Kanon . Siden arbeidede han nesten hele sit Liv igjennem med dette Mordredstab . Da er det bedre , at vi tidligt mode Herren ; da kunne vi ikke , sna lenge vi leve , snoe os fra Hans Dragelse . Ja , om vi end fenere fare vild en Stund , om end Livets <b>Sand</b> og Stov legger fig paa hine Sedekorn , faa er der dog fremdeles Liv i dem , og i sin Tid komme dog de forst op. Intet har oftere fort forvildede og i Elendighet » nedfunkne Syndere paa Omvendelsens Vei end Tilbngeblikket pna en gudfrygtig og lykkelig Ungdom og Ihukommelsen af de Stykker af Guds Ord , forn de hnude lert i sin Barndom . Ja , nettop de ere for Mange blevne en Kjep og Stav i Dodsstyggens Dal . — Men hellerikke for Eder Foreldre bliver et faadcmt Arbeide paa Eders Borns Frelse uden Frugt . Kun den Fader og <b>Moder</b> besidder i Nornenes <b>Oine</b> den rette Verdighed , som har den himmelske Fader staaende bagved sig , — som forretter sit Kald i Hans Navn og henviser sine Born til Hnm . Da hviler der en Glands as guddommelig Majestet om deres Hoveder , som frembringer en langt storre og en sannere Lydighet » end al blodagtig <b>Kjerlighed</b> og alle strenge Befalinger . Dette er imidlertid en Frugt , forn falder mere udåd , medens derimod en anden falder i dit eget <b>Hjerte</b> . Io eenfoldigere vi tåge Guds Ord , desto storre Magt har det . Hvo forn felv er ret liden og eenfoldig , idet han meddeler sine Born det , hvo som nedlater sig til dem , ligesom

<h4>409</h4>Naar nndtages Foreldre og Born saavelsom 3 - Egtefolk , har Gud ikke stillet Nogen i et nermere og inderligere Forhold til hincmden end Sostende . Een Fader har avlet dem ; een <b>Moder</b> har baaret dem under sit <b>Hjerte</b> ; een <b>Kjerlighed</b> har bestjermet og pleiet dem ; fra de famme Hjerter er der opsteget Bonner for dem til Naadens Throne . Under eet Tag have de boet ; ved eet Bord have de spiist , og i een Have have de leget med hincmden . I mange Aar har Samlivets Vane omslynget dem ; i Ungdommen , da det ene <b>Hjerte</b> faa let hengiver sig til det andet , have de ved Guds Orden veret henviste til hincmden . Samme Huusstel , samme Opdragelse, samme Skole , samme Arbeide , samme Gleder bor have gjort dem saagodtsom til Eet . Man skulde vesten troe , at Sostende maatte vere fastere forenede end Wgtestabet stiftes ved eget Valg , Sostendeforholdet ved Guds Orden alene . Wgtestabet indgaaes ofte af fyndig Egenkjerlighed; det kan ogsaa vere en overilet Gjerning ; med Sostendepagten kan dettte aldrig vere Tilfeldet . For den ligger Guds Orden til Grund , og dertil kommer en heel Ungdoms Vane . Allerede af hvad hidtil er sagt seer du , at Sostendeforholdet maa have et hoit Verd i Guds <b>Oine</b> . Men stue vi ind i den hellige Historie , faa bliver dette endnu klarere . lefus Christus er bleven vor Broder . Han har ikke stammet sig ved dette Navn og denne Stilling til os . Ved at fremtrede som vor Broder har han adlet Sostendestanden . — Derhos vidste Han ikke at sette sin af Ham gjenfodte Menighet) i et inderligere gjensidig ! Forhold end det broderlige . I Forhold til Faderen ere vi Born ; i Forhold til Sonnen er Menigheden og Sjelen Bruden ; i Forhold til hverandre ere vi Brodre og Sostre . Apostlene hilse Menighedernes Medlemmer som Brodre og Ssstre . Brodre i Christo er Udtrytket for den inderligste Eenhet » i Troen . Menighedslemmerne kysse hverandre med Broderkysset . Denne Vroderkjerlighed slynger sig som et Fuldkommenhedens Bacmd om den hele store Menighet » . Her sinder ingen Undtagelse Sted . Paulus anbefaler felv den stakkels Trel Onesimus til Filemon i Kolosse som sin kjere Broder . Et Broderstad har nltsnn Herren stiftet her pnn Jorden . Om dette synge Englene paa Marken ved Bethlehem med de Ord : „ Fred paa Jorden ! " og Herren <b>selv</b> kalder sig vor Broder . Hvad bor nu Sostende efter Guds Raad vere for hinanden? Fodte , fclvstrcvne Venner , hvilke Gud felv har medgivet os , hvilke man ikke forst behover at erhverve sig . Ligesom der paa en Gran voxer flere Grene ud af Stammen paa eet Punkt , faaledes ere ogsaa Sostende jevnsides fremspirede

<h4>800</h4>Troen er tilveiebragt . Hvad stal hau uu gjore , for at Guds Ord ogsaa kan vinde Herredommet hos hans <b>Hustru</b> og Christi <b>Kjerlighed</b> indtage heudes <b>Hjerte</b> ? — Deels bor han med rolig og venlig Fasthed fortsette den engang indforte Orden og tillige ved passende Anledninger belere , formane og soge at overtale hende , men fremfor Alt hver Dag inderligt bede til Gild om sine egne Synders Forlatelse og om at Hnn vil opvekke hans Hustrues Sjel . — Eu gudfrygtig uug Hacmducerker giftede sig med en Pige , som havde et godt Lov for sin Netstaffenhed , men forn endnu vaklede , hid og did mellem Herren og Verden . Dette vidste han , men mente , at han i Z-Egtestabet snart skulde vinde hende for Herrens stille Efterfolgelse . Heri havde han dog aldeles forregnet sig . Efter at de havde veret gifte en Stuud iudface han , at hans Bestrebelser vare frugteslose . Hau havde spendt sig i Ang sammen med en Vantro . Hun vilde ikke lade sig forstyrre i sin Vantro og sit verdslige Vesen . Begge bleve saa aldeles ueeus , at det nesten daglig kom til de heftigste Ovtrin . Taug Mcmden stille , faa var huu fortonet over hans Taushet » , og talte han , faa var huu fortonet over hans Tale . Hendes Trettekjerhed gik over alle Grcendser . I denne Nod greb han til det rette Vaaben . Han bekjendte forst for Herren den Formastelfe , han havde begacet ved at velge en faadcm <b>Hustru</b> , og bad Ham dernest om deil rette Viisdom til med Evangeliets Vaaben at boie hendes vilde Sint » . Saasnnrt nu Strideus Lue atter vilde blusse op , tog han sin <b>Kone</b> venligt i Hcumden og sagde : „ Kjcere <b>Kone</b> ! jeg vil forst gaae hen og bede for dig , og faa kuuue vi tales uermere ved om Sagen . " Han gik , og saaledes gjorde han , huergcmg hun pacmy vilde give sit onde Sint » Luft . Derhos fandt han sig med Taalmodighed og Sagtmodighed i hendes Adferd og vandrede nu som for i Herrens Frygt for hendes <b>Oine</b> . Disse tause Vnnben fra Herrens Nnstknmmer vare for sterke , for sterke <b>selv</b> for hendes ustyrlige Acmd . Huu gik i sig felv og blev en stille , ydmyg , gudfrygtig , trofast <b>Hustru</b> . Hun tilstod siden for sin Mcmd , at de Ord , han hvergcmg fagte : „ leg maa forst bede for dig , " havde gacet hende saaledes til <b>Hjerte</b> , at hun efterhacmden haute lert at kjende sig <b>selv</b> , sin Gud og tillige sin Mands <b>Kjerlighed</b> og Trofasthed . — Dog , langt oftere end Mentene befinte Konene sig i en saadan Trengsel . Ligesom i Herrens Tid , saaledes gribe de endnu ofte den Dag idag Ordet om Korset baade hurtigere og med et dybcre Siud eud Mentene . Nu er der jo lugen , som eu trofast <b>Hustru</b> hellere onster at drage med sig md i denne Salighet» end den hende af Gud betroede Maud . Men desverre

<h4>825</h4>herske , er der bestandig Ufred ; et godt Raad , en venlig Hjelp og Tjeneste er derimod stedse velkommen . Derhos bor enhver Svigermoder tenke flittigt paa at Herren setter hende ud af Virkfomhed , for at hun , inden hendes sidste Time kommer , allerede her stal indgace til Guds Hvile . Salig er den , hvem Gud efter Dagens Slid og Sleb stjenker en rolig og stjon Aften , hvori han kan overskue Dagen , soge Naade for sine Synder , prise Gud og forberede sig til den Nat , da Ingen kan arbeide . — Fra Mcmtens <b>Moder</b> komme vi til Konens Foreldre . Med Formaning til hjertelig Idmyghed bor de ndsende sin Datter til hendes nye Hnus . De bor ikke forlange , at Alting der stal vere ordnet og indrettet forn i deres eget Huus . Det er Mcmtens Huus . Den unge <b>Kone</b> bor anerkjende og ere den Stik , forn hersker der , forsaavidt som den ikke er syndig . Hun er en Kvist , som er afstaaren af deres Huus ' s Stamme og indpodet i en anden . Podekvisten kan ikke medbringe den gamle Stammes Rodder ; den bliver fammenfoet med en ny og sat i Forbindelse med dens Rodder . Og fremfor Alt bor den unge Kones Foreldre vogte sig for at ville holde hendes <b>Hjerte</b> tilbage i sit eget Huus og anvise hende en los Stilling midt imellem begge Familier . Hun har jo hengivet sig ganske til sin Mand , og hendes Foreldre have <b>selv</b> givet ham hende . — Der gives intet Hnus , som ikke har sin Sorg og Smerte . Ogsaa med den er hun sammenviet. Foreldrenes Huus bor ikke vere noget Sted , hvor huu kan gaae hen og udose siue Klager og hvorfra hun vender tilbage til sit eget Huus som fra sit rette Hjem . I hvilken Glands staae ikke de Gudfrygtige i den gamle Pagt , ja endog for den gamle Pagt , ved Siden af mange Christne i vor Tid ! Bethuel og hans Born betragte Rebekka som virkelig stilt fra deres Stamme ved sit Giftermaal med Isak . Ved Afsteden sige de ikke et Ord om at hun stulde kunne soge nogen Tilslugt i sive Foreldres Huus . Hun maa tåge sit hele <b>Hjerte</b> med sig til det fremmede Land og det nye Hjem . Den unge Tobias ' Z Svigerforeldre , Nagnel og hans <b>Hustru</b> Edna , formane sin Datter ved Afsteden til at ere sin Mcmds Foreldre som sine egne Foreldre . Ikkedestomindre kan og stal dog den uuge Koues <b>Kjerlighed</b> og HErefrygt for sine Foreldre fremdeles vedvare . Ja . dette vil i saa meget hoiere Grad vere Tilfeldet , nåar de med sin Viisdom og Erfaring bidrage til at berede hende Sted i hendes nye Hjem . Hun er nu forst og fremst sin Mcmds <b>Hustru</b> og saa dernest deres Varn . — For det Tredie bor Mcmden med <b>saud</b> Klogstab arbeide paa at bevare et godt og fredeligt Forhold mellem Svigermoder og Svigerdatter .

<h4>829</h4>Navnet „ Moter " bor ikke vere nogen blot Talemaade i Svigerdatterens Mnnd . Denne <b>Moder</b> har hun at takke for sin Mand ; hun har baaret ham nnder sit <b>Hjerte</b> , bragt ham til Verden med Smerte og opdraget ham med Moie . Meget af det , som den unge <b>Kone</b> elsker hos ham , har nettop dcnue <b>Moder</b> indplautet ham . — Har Svigermoderen sine haarde og underlige Sider , saa stal Svigerdatteren kun ret ofte gjenlage Navnet „ <b>Moder</b> . " Om hendes egen <b>Moder</b> havde den samme Sindsbestaffenhet » , vilde hun ikke opsette sig mod hende . Her er ogsaa en <b>sand</b> <b>Moder</b> ; thi 2Egtestabet er en <b>sand</b> Forening . — Derfor bor <b>Konen</b> ogfaa vel vogte sig for at afdrage sin Mcmds <b>Hjerte</b> fra hans <b>Moder</b> . Deraf vil en <b>Hustru</b> aldrig hoste Velsignelse . Hvo som giver Anledning til Overtredelse as det fjerde Vnd , arbeider med det Samme paa Overtredelsen af det fjette . Budene ere ti Grene paa een Stamme . Naar een af disse Grene fortorres , doe ogsaa de andre ud ; thi hele Treet er da sygt . Ingeu <b>Hustru</b> maa indbilde sig at kuuue fastholde den Mcmds <b>Kjerlighed</b> , som hnn har gjort utro mod sin <b>Moder</b> . Den , som lonner den <b>Kjerlighed</b> med Utak , som har fnlgt ham fra Livets forste Morgen til denne Dag , han vil endnn snarere vende den <b>Kjerlighed</b> Nyggen , som forst fenere er indtraadt i hans Liv . Den , forn er bleven utro mod den medfodte <b>Kjerlighed</b> , bliver det ogsaa mod den egtestabelige . — For det Tredie bor eu ung <b>Kone</b> vel vogte sig for at beklage sig utenfor sit Huus . <b>Selv</b> til sin egen <b>Moder</b> og sine egne Sostende bor hun ikke sige noget Ufordeelagtigt om sin Svigermoder . Har hnn Noget at klage over , kan hun ikke bere det hos sig <b>selv</b> , saa klage hun sor Gud Herren !

<h4>852</h4>san er ogsnn den ovrige Historie saa meget rigere paa gruovvcekkende Exempler paa dem . Allerede de greske Sagn skildre nogle Stedmodre med de sorteste Farver . Romerne havde det Ordsprog : „ at klage for en Stedmoder ; " det vil sige : at klage uden Nytte . Den romerske Keiserinde Agrippina, Clcmdius ^ s Gemalinte , forgiftede sin Stedson Britcmuicns for at bringe sin egen Son Nero paa Thronen . Hun fik imidlertid eit skrekkelig Lon til Tak derfor . I Eventyrene forekomme mange Stedmodre af det verste Slags . De indbragte Born maa vige Pladsen for deres egne ; de blive enten drebte eller jagede i Landflygtighed . Mcmgesteds hores Fortellinger om at Sorgen for de efterladte Smaae har fulgt de afdote <b>Modre</b> i Grnveu og i Himmelen . I Nattens Stilhed stulle de have kommet tilbage , givet sine Smaae <b>Die</b> , kjemmet deres Håar og vedet det med bitre Taarer . Desverre taler ogsaa Historien lige til den Dag idag hoit nok om Koner , som vel bare Navn af Moore , men dog ikke havde noget moderligt <b>Hjerte</b> . O du sode Nnvn , hvori Gud har indesluttet Liv , <b>Kjerlighed</b> og Trofasthed ! ogfaa dig kan Synden gjore til lis ; ogsnn nf dig kan der istedetfor Melk dryppe Edik og Galde . Ja , faadcm Magt har Synden ! Hvorledes befeire nu reistane Stedforeldre hine Hindringer? Hvorledes bevare de sine Sjele , saa de ikke foie nye Sutte og Taarer til dem , deu Tabtes dyrebare Navn allerede hnr fremkaldt ? Ved at annamme Bornene i Jesu Christi Navn . Gud Herreu er deu , soul efter siv Viisdom skjenker eller negtcr 2 - Egtefolk Born . Born ere Herrens Gave ; Livsfrugt er en Lon . Gud aleue stjenker tjudelige Born , og den famme Gud stjenker ogsaa Stedborn . Gid alle Stedforeldre ret vilde betenke : „ Disse Born ere en Guds Gave til mig ; Han har betroet mig dem ; dermed har Han ogsaa betroet mig noget Stort , og Han vil engang treve dem af min Haand . Den Gud , som har cmtaget det Folk , som ikke var Hans Folk , til sit Folk , som har gjort Mennesker , der ikke vare Hans Born , til sine Born , — den Gud , forn uden nogen Fortjeneste eller Verdighet » pnn min Side ogfaa har cmtaget mig arme , fortabte Synder til fit Varn , Han har givet mig Born , forn ikke vare mine Born . " Betragt nu daglig diue Stedborn med saadcmne <b>Oine</b> . Og koster " det dig Moie at outugte dem , san glem ikke , at din egen Ovtngtelse har voldt og vil volde din Gud lcmgt storre Moie . — Bed deruest daglig din Gud og Herre om egte Fader- eller Modertjerlighed til disse Born . Al <b>saud</b> <b>Kjerlighed</b> kommer fra deu evige <b>Kjerlighed</b> og erholdes kun ved Bon . Endnn har aldrig noget oprigtigt <b>Hjerte</b> forgjeves bautet paa deuue Naadeport . Gud stjeuter helst , hvad Han

<h4>884</h4>mine gamle Foreldre ; jeg er deres Forforgerinde , deres Slotte ; Gud hnr fat mig i dette dyrebare Kald " , saa er dette en Grund , som har en god Klang i Guds Oren . — Det hender ogsaa , at Piger , som Herren ved Doden har frataget deres Forlovede , forblive denne troe i sit hele Liv og aldrig ville indgace nogen ny Forbindelse . Og endelig sindes der , stjondt ikke ofte , Saadcmne , forn troe at knnne tjene Herren bedre i sit Kald som ugifte end forn gifte . Ligesom enhver Stand , saaledes har nu ogsaa det ugifte Liv sive Farer . Vi ville her ganske sette dem ud af Betragtning, forn forblive eenlige for ikke at opgive sin kjodelige Frihed og sin Bekvemmelighet » . Men vi ville navnligen niinde vore Lesere om hvor ofte ugifte Personer lige til en hoi Alder vedblive at elste Tant og Forfengelighed . Andre forherde sig stedse mere i sin Selvraadighed og Egensindighet); deres seregne syndige Natur bliver ikke boiet ved en daglig Opofrelse for <b>Hustru</b> eller Maud , men cmtager en skarp , udpreget Skikkelse . Have de intet bestemt Kald , saa udfylde de ofte sin Tid med Sladder og andre Tungesynder . Hertil kommer Klager over deres eensomme Stilling , Klager over enhver liden Gjenvordighet » ; Gud har behandlet dem uretferdigt ; de see med Bitterhet » paa sine Sostende , der leve i et lykkeligt Familieliv ; de ere starve i sine Domme og blive baade Andre og sig <b>selv</b> til Byrde . Deres hele Tilverelse dreier sig om dem <b>selv</b> . Hvor vidt forstjellige saadcmne Mennesker ere fra de Ugifte , Apostlene har for <b>Oie</b> i vor Text , behove vi ikke nermere at paapege . — Hvad Raad er der nu herimod ? Hvorledes stulle de kunne bevare sig <b>selv</b> for de nevnte Forvildelser ? Herpaa give vi det tredobbelte Svar : „ Derved , at de gribe Herren i en barnlig Tro , velge sig et bestemt Kald og slutte sig til en Familie . " Den daglige Omgang med Herren holder ogsaa de Eenliges Hjerter friste , glade , deeltagende og tilgjengelige. Den modarbeiter Selvraadigheden , Egensindigheten og Forstenelsen i egne Tanker , Synsmaader og Luner . Den holder <b>Kjerligheden</b> ilive , saa at man beder for Andre , meddeler Andre og virker for Andre . Vi have kjendt eenlige Personer , som i den stille Omgang med sin dyrebare Sjeleven besad en Rigdom paa <b>Kjerlighed</b> , Venlighet » , Idmyghed og Taalmodighed , som man ikke stulde have ventet at finte i deres übekjendte Vråa . Jesus Christus er den , som allevegne beseirer vort <b>Hjertes</b> Forvcndthed , Trodsighed , Haardhed og Bitterhed og gyder himmelsk Venlighet » og opofrende <b>Kjerlighed</b> deri . Hvo som omgaces med Ham , besidder den rette Ven , Broder , Maud , Kjep og Stav . — Vi gaae uu over til Virkekredsen og Familien . Den katholste

<h4>1025</h4>De Fattige derimod gaae ind ad den og lade sig gjore rige af Herren . Istedetfor at knurre over din Fattigdom bor du heller takke Herren for at Han har bygget din Hytte saa ner ved Frelsens Port , og at Hans Indbydelser lyde saa klart og tydelig ! derind til dig . Hold du kun fast ved din Frelser . Vederkveg du dig og Dine med Guds gode Ord . O , hvor deilige blive itte Aftnerne i den fattige Hytte , nåar Faderen sidder og leser hoit i <b>Sandhedens</b> evige Ord , og <b>Kone</b> og Born begjerlige og glade lytte derpaa ! Da ovfyltes den lille Menighed med den Hellig-Acmd ; Hjerterne uduide sig ; den dunkle Lampe brender klarere ; Fattigdommen glemmes ; I fryde Eder i Herren og Hans Styrke ; Guds <b>Kjerlighed</b> er udost over Eder ; I fole og ane , at I have en usigeligt riig Arv . — Lad derhos Intetsomhelst stjele de velsignede Sondage fra dig . Du veed , at de fremfor alle andre Dage ere rige paa Herrens Naade . Paa dem skinner det evige Livs Morgenrode ind i din Hytte ; de ere dig et Pant paa den store Sabbat , paa den store Hvile hos Gud . Sondagen udsoner dig med alle cmdre Stender . Alle Stender i Menigheden hore det samme Ords Forkynnelse , have den samme Frelser , ydmyge sig for Gud i den samme Syndsbekjendelse , blive retferdiggjorte ved den samme Christi Fortjeneste , ere deelagtige i det samme Christi Legeme og Blod og have det samme evige Livs Haab . Da gaacr den Opstandnes Nost gjennem Menigheden og raaber : „ Fred vere med Eder ! I mig er der hverken Mand eller <b>Kvinde</b> , hverken lote eller Greker , hverken Trel eller Fri , hverken Fattig eller Riig . De , som ere i Christo , ere nye Stabninger . " — Med Hensyn til dit Himsvcesen , da ver tilfreds med hvad Herren stjenker dig . Ingens Liv bestacer i hans Gods , i det , at han har Overflod . Eier du Idmyghed og Bestedenhed , fan er dn ligesnn riig som den Nige . Den Nige kan hellerikke mere end spise sig met ; han kan hellerikke boe i mere end eet Huus ; han berer hellerikke mere end een Kledning ad Gangen ; han sover hellerikke i mere end een Seng ; han faner tilsidst hellerikke mere end een Liigkiste og een Grnv . — Er det muligt , sna lacm ikke ; indstrenk dig heller , saa meget du kan . Ja , bed heller et christelig . <b>Hjerte</b> om en Understottelse , forent ) du gaacr hen og lacmer . Lacm gjor Lognere og letsindige Mennesker . Nnnr Dagen kommer , at man stal betale tilbage , og man ingen Penge har , undseer man sig for at tilstaae dette for sin Gjeldner . Af Hovmod bliver man en Logner , tager Skade paa sin Samvittighet » , synder <b>selv</b> og frister ogsaa den Anden til Synd . Vogt dig sor Laaneindretningerne ; i de fleste Tilfelter ere de Odeleg-

<h4>1229</h4>af Guds Dom over ham , har langt vanskeligere for at komme til en <b>sand</b> Omvendelse , det er Noget , som settes ud af Betragtning . — Man sorger og er utrostelig , nåar et Barn nedsynker sna dybt i Uredelighet » , Tyveri , Bedrageri , Falskneri og Deslige , nt den verdslige Ovrighed mna udstrette sin Arm efter ham og give ham hans velfortjente Lon ien Strafcmstalt . Men <b>selv</b> da er det dog itte saa meget Synden som Skammen , itte saa meget det forstyrrede indre Liv fom den forstyrrede jordiste Lobebcme , der bringer Sorg over Sjelen . — Noget af det , der volder en Familie den storste Hjertesorg , er et Barns uheldige Giftermaal, idet Parret itte kommer godt overens , idet den nnge Kones Vei gaacr igjennem Gracidsdalen , idet begge 2 Egtefeller mane kjempe med Trang og Fattigdom . Hvor mange Sukke og Klager over eu saadan Lod maa itte en Geistlig hore af nedborede <b>Modre</b> ! Men hvor fjelden klage de ikte over Aarsagen til denne Elendighet » ! Hvor fjelden sukker man ikke over den dybeste Nod i Huset , nemlig over at Guds Ords Lys aldrig er blevet tendt og aldrig har brendt der ! — Endelig ville vi ogsnn omtale den meest bagvendte Sorg , der kan tenkes i Verden . Endnu i vore Dage gives der Fedre og <b>Modre</b> , som itte kunue finde sig i at deres Son eller Datter er kommen til en levende Tro . Naar de ville udtrytte sig rigtigt mildt , klage de over nt den unge Mand , Pige eller <b>Kone</b> er bleven saa ensidig , at de negte sig Livets Gleder og give Slip paa saa mange fordelagtige Forbindelser . Herpaa svare vi ikke et Ord . Saaledes kunue alene de tale , som Intet kjende til Guds Boms Lyksalighed, Intet til den hoieste Forbindelse , der kan gives her i Verden , Intet til Guds unådige Forjettelser . — Hvor ligger den rette Anrsng til Sorgen ? Hvad sorger vor Herre og Frelser over ? Nnar Han udenfor Jerusalem felder bitre Taarer over Guds gamle Stad , uaar Hans <b>Hjerte</b> ved den Leilighed bolger og bruser som et dybt oprort Hav , uaar Hans Sjel i Gethsemcme er bedrovet indtil Doden , da er det itte Folkets Armod , som nedboier Ham , da er det itte Tanken om den Mengde Fattige , Enker , Faderlose , Kroblinger og Elendige , der er i Landet , som nedtrykker Ham , ligesaalidtsom Synet nf de romerske Faner , der vaie over Hans Fader Davids Borg ; nei , det er det gamle udvnlgte Folks Synder , Vantro , Frafald , der hviler faa tungt paa Ham og fremknlder Taarerne af Hans <b>Oine</b> ; det er itte en enkell af Folkets Synder , hvorved det maastee iser har styrtet sig i Ulykke , som smerter Ham saa dybt ; nei , det er Israels samtlige Synder , al dets Ugudelighet ) , dets hele sorgelige indvortes Tilstand . — Lerer dog at sorge over Elendighetens Rod !

<h4>1502</h4>Til alle Tider have Menneskene sorget over sine Afdote . Abraham begreder sin trofaste <b>Hustru</b> , som havde fulgt hnm paa hnns moifommelige Vandringer fra Kaldea til Mesovotamien, til Kcma ' cms Land og 3. Egypten . Josef begreder sin gamle Fader , Jakob , og tilligemed ham holde 3. Egypterne et stort Klagemaal ved hans Grav . Israels Born begrcete Moses i tredive Dage paa Monbs ode Mnrker . Ta deu gudfrygtige Kong lostns , den sidste verdige Wtling nf David pnn hans Kongethrone , var falden ved Hadadrimmon i Megiddos Dal , holdt Folket en Veklage over hnm , som gjenlod lenge bngefter , idet ingen Konge som han senere opsteg paa hans Fedres Throne . Gane vi over til det Nye Testamente og trede ind i den nfdode Lazarns ^ s eller i lain » Xs Huus eller mode Ligtoget i Nerhedeu af Nain , faa lyde Klagemaal os overalt imode . Og det Samme har veret Tilfeldet hos alle hedenske Folkeslag ; <b>selv</b> hos de dybest snnine af dem findes der Spor af Sorg og Klage over de Afdote . Denne Sorg bor ogfaa vedblive i Christi Kirke . Det man vere letsindige Mennesker , som kuune stnne ved sin Faders eller Moders Dodsleie , Ligkiste og Grnu udeu uogen smertelig Folelse nf hvad Gnd i dem havde skjenket dem og nf deil <b>Kjerlighed</b> , forn de med dem tale her paa lordeu . Tet maa vere en elendig og letsindig ZEgtemand , som paa nordamerikansk Viis allerede ved sin forste Hustrnes Jordfestelse kan overveie , hvem der stal blive hans anden <b>Kone</b> . Vco vore Nerbeflegtedes Don tnler Gud Herreu snn krnftigt og indtrengente til os , nt denne Tale nok bor gjenlyde eu god Tid bngefter og gjore os vort store Tnb foleligt . Vi ville uu for det Forste gjore os <b>selv</b> Nede for hvem vi sorge over . Vi pleie at sige : „ leg sorger for min Fader, min <b>Moder</b> , min <b>Kone</b> , min Mand , mit Barn , min Broder , min Soster . " Men dette er kim i uisse Tilfelder fcmdt . Ere de vandrede bort i Vantro og Ugudelighet » , have de lige til sin sidste Stnnd veret hildede i Satans Lenker , da mane vi sorge sor dem ; thi saauidt vi forstnne , ere de da dode en dobbelt Dod . Ak , da er vort <b>Hjerte</b> sonderttmst ; da ndgyde vi bitre Taarer ; da kan i Grnnden heller Ingen troste os . Og Sorgen bliver saa meget dybere , som vi jo ved enhver saadan Leilighet ) maae sige til os <b>selv</b> : „ Tv har ikle arbeidet paa denne Sjel med al den christelige KXrlighed og Trostab , som du burde . " Dcnue Sorg ophorer ikte , foreud vi have facet alle vore Synders Forlatelse og , fnldendte i Herren , lere at betragte Historien , altsaa tillige de Fortabtes Historie , med den guddommelige Snudheds , Netferdigheds og Hclligheds <b>Oie</b> . — Er derimod Nogen hensove! i Herren , snn sorge vi itte for hnm , men for os <b>selv</b> .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012052204016'target=_>Ahlfeld, Friedrich, 1869, Menneskelivet i Guds Ords Lys</a><h4>295</h4>saaledes vil det ogsaa gaae med dine Born . I den tidlige Barndom er Hjertet blodt og Hukommelsen ung . Hvad man da loner , det indprceger sig dybt og glemmes fordetmeste aldrig . Men ikke nok hermed . Hvad vi loere i vore forfte Barneaar , og navnlig af vor <b>Moder</b> , det feer anderledes ud end hvad vi senere lcere . Morgenduggen hcenger ved det ; Moderkjcerligheden med den hele Erindring om disse lykkelige , sorgfrie Aar omsvcever det . Naar vi senere trceffe saadanne Udsagn og Fortoellinger igjen , tale de et ganske andet Sprog end de , vi siden have lcert . Dette ligger Me i Guds Ord , som om nogen Deel af samme var kraftigero end en anden . Det ligger i Tiden og Hjertets Modtagelig ? hed . Disse fyrste Udsagn og Fortcellinger have flaaet Rodder i hele Livet . Saaledes er det ogsaa med Sceden ude paa Ågeren . Hvad der saaes tidligt om Vaaren , fcefter dybeft Nod . Om ogsaa en anden Ager senere besaaes med samme Sced , modstaaer hiin dog bedre Torken og lonner Landmanden med en rigere Afgrode . Hvad der fra forst af er indprceget i Barnets <b>Hjerte</b> , giver dets Liv dets Retning . Historien kjender en Mand , som allerede i sine tidligste Aar legede med ' en Kanon . Siden arbeidede han ncesten hele sit Liv igjennem med dette Mordredstab . Da er det bedre , at vi tidligt mode Herren ; da kunne vi ikke vride os fra Hans Dragelfe, saalcenge vi leve . Ja , om vi end senere fare vild en Stund , om end Livets <b>Sand</b> og Stov lcegger sig vaa hine Scedekorn , saa er der dog fremdeles Liv i dem , og i nn Tid komme dog de forst op. Intet har oftere fort forvildede og i Elendighed nedsuNkne Synders paa Omvendelsens Vei end Tilbageblikket paa en gudfrygtig og lykkelig Ungdom og Ihukommelsen af de Stytter af Guds Ord , fom de havde lcert i sin Barndom . Ja , nettop de ere for Mange blevne en Kjcep og Stav i Dodsfiyggens Dal . Men hellerikke for Eder Forceldre bliver et saadant Arbeide paa Eders Vorns Frelse uden Frugt . Kunden Fader og <b>Moder</b> bestoder i Bornenes Vine den rette Vcerdighed , som har den himmelske Fader staaende bagved sig , som forretter sit Kald i Hans Navn og henviser sine Bprn til Ham . Da hviler der , en Glanos af guddommelig Majestcet om deres Hoveder , som fremkalder en langt storre ZErefrygt og en langt sandere Lydighed end al blodagtig <b>Kjoerlighed</b> og alle strenge Befalinger . Dette erimidlertid en Frugt , fom fald er mere udåd , medens derimod en anden falder i dit eget <b>Hjerte</b> . Io eenfoldigere vi tåge Guds Ord , desto stprre Magt har det . Hvo fom felv er ret liden og eenfoldig , idet han meddeler sine Born det , hvo fom nedlader sig til dem , ligesom Gud Herren i Ordet <b>selv</b> har nedladt sig til os , han mod-

<h4>530</h4>lagde : „ <b>Mor</b> , vil du da ikte give os Frokost idag ? vi ere sultne . " Men Moderen gav intet Svar . Da bankede de omsider vaa Doren til den noermeste Nabokone med den Bon : „ O , kom over til os ; <b>Mor</b> vil slet ikke vaagne . " Og hun kom og saae og sagde til Bornene : „ Kjcere Born ! Eders <b>Moder</b> vaagner ikke mere i dette Liv ; Herren har kaldt hende bort ; kom og folg med over til mig ! " Da bliver der Sorg og Smerte i Barnesjelen ; da seer det ud derinde , som nåar et Haggelveier gaaer over en Have , medens den staaer i sit lyse Vaargront . Men <b>selv</b> hvor Elendigheden ikke har brudt saa voldsomt og uventet ind , have vi dog mangengang seet en übeskrivelig Smerte afmalet vaa de unge Ansigter , nåar den sidste af Forceldrene blev baaren til Graven . O , hvad vil det sige at odelcegge et Nede for Fuglene ? De pibe og klage nogle Dage vaa den tomme Green og bygge sig dervaa et nyt istedet . Forceldre , virtelige Forceldre faaer Barnet aldrig igjen . Skjcenker end Gud det noksaa trofaste Pleieforceldre , saa faaer det det dog ingen <b>Moder</b> mere , som har baaret det under sit <b>Hjerte</b> . Med Forceldrene var Vsrnene efter Guds Orden sammenvoxede; Andre derimod maae de vcenne sig til . Og desuden forbliver den lille Flok ofte ikke engang samlet . Eet sinder her , et andet der sine Forbarmere og Forsorgere . Og hvor kummerligt bliver der ikke ofte sorget for dem , navnlig i Henseende til <b>Kjcerlighed</b> ! Dog , forst komme vi til det Sporgsmaal : „ Hvorfor gjor Gud Herren saadcmne Ting ? " Kjcere Lcesere ! hertil maae vi forst sige : „ Han vil ikke gjerne have slige Sporgsmaale , og Han besvarer dem fordetmeste slet ikte i Tiden . Ofte forstaae vi senerehen Noget af Hans underlige Forelser ; ofte lader Han os ogfaa vente til hiin store Dag , da alle Svorgsmaale stulle blive besvarede , eller da vi ikke Icenger forlange noget Svar , fordi vi nu have stuet klarere ind i Hans Raad . " Ikke destomindre ville vi give et Svar i al Skrobelighed . En gammel from Christen sammenligner Gud Herren med en klog Gartner , som studser et Trce og afhugger de store Grene , der brede sig ud over de unge Rodstud , for at ogfaa disse skulle faae Godt af Solen og umiddelbart bestinnes af den . Saaledes tåger ogsaa Gud Forceldrene de store Grene , som bredte sig ud over Nornene , bort , for at disfe skulle stue lige op i Naadesolen. De stulle tidligt drives til Herren og lcere at troe , bede , omgaaes med Ham og scette sin Lid til Ham . Der er <b>Sandhed</b> i dette Svar . Ak , hvor ganske anderledes have ikke de Forceldrelose da lcert at bede ! Hvor ganske anderledes har da ikke Fadervoret gaaet fra deres <b>Hjerte</b> og over

<h4>538</h4>deres Lceber ! For det Andet tåger Gud ofte Forceldrene bort , for at de ikte stulle fordcrrve siue Born med siu Vantro og sine Synder . Nige og overdaadige Folks Born have ofte nnstet sine Forceldre , for at de ikte skulde drages ind i Verdens Overdaadighed og Ugudelighed . Gud har bragt dem i et Huus , hvor de have lcert at arbeide , bede og soge Hans Ansigt . Born as fattige Forceldre , men som dog Intet vidsts om Frelsen og Frelsens Vei , har Han ofte bragt i et troende Huus eller en Ovdragelfescmstalt . En gndfrygtig Pleiefader og Pleiemoder , en gudfrygtig Forstander eller Forstanderinde have da gjort imod Bornene , hvad deres Forceldre itke havde gjort . De have fort dem til Herren . Ovfostringsanstalten eller Pleieforceldrenes Huus er blevet dem et nyt Livs Vugge . Mange fattige Born have i en modnere Alder bekjendt for Gud : „ Min Fader i Himlene ! da jeg mistede min kjodelige Fader , fandt jeg Dig . " Og endelig for det Tredie lader Gud mange Born blive forceldrelose , for at Menigheden stal blive deres <b>Moder</b> . O , dersom den ogsaa isandhed blev det ! O , dersom den blot stedse i <b>sand</b> Menighedsfolelfe bekjendte : „ Det er vore Born ; vi maae vcere dem i Forceldres Sted ! " Men Gnd hjcelpe os ! hvorledes blive Me de Forceldrelose ofte behandlede ! Vi vide , at de stakkels Born i enkelte Menigheder ere blevne såtte til formelig Licitation og overdragne til de Mindstbydende . Man har behcmdlet deres Opdragelse vaa samme Maade som et eller andet übetydeligt jordisk Anliggende , som en Vei , der skulde istandscettes , som en Dam , ' der skulde ovfores. Man har ikte bekymret sig om til hvem Bornene kom , hvorledes de blev ovdragne , hvortil de blev brugte . Hvor mange Forceldrelose have ikke maattet soge at vcekke Medlidenhed for sig <b>selv</b> og sine Pleieforeldre og gaae for Hvermands Dor ! Ofte har Menigheden viist sig som en ugudelig Stedmoder mod de stakkels Born , og i hele Aar have deres Formyndere ikle erkyndiget sig om dem . Naar Forceldrene doe , bor de ncermeste Slcegtninger , Forceldrenes Sostende eller andre Paarorende , Fadderne eller Formynderne tåge sig af deres Born . Hvad er ellers Elcegtstabet til ? Naar et Lam mister sin <b>Moder</b> , give de andre Faar iHjorden oet forladte <b>Die</b> tilligemed sine egne . Ncesten blandt alle Folkeslag ganer der Sagn om at en Hin eller endog en Utvinde har givet udsatte orn <b>Die</b> og er bleven deres Pleiemoder . Ere da ikte I meget mere end de ? Bor der ikke boe en ganske anden <b>Kjcerlighed</b> i Guds Born end i de stakkels fornuftslose Skabninger ? I ligegvldige og ukjcerlige Menighedsforstandere og Slcegtninger , og I Mammonstrcelle , som udstrcette Eders Haand ester de Forceldrelofes Gods ,

<h4>974</h4>Herre Jesus Christus , lad aldrig Brud og Brudgom , lad aldrig Mand og <b>Kvinde</b> glemme , at Du er deres fprste Brudgom . Du har elstet os med en evig <b>Kjcerlighed</b> og draget os med Mistundhed . Du har beitet til os med en <b>Kjcerlighed</b> , hvis Lige aldrig er avkommet i noget Menneskes <b>Hjerte</b> . I den hellige Daab såtte Du Trolovelsesringen paa vor Finger . Med din tilgivende , sogende , staanende og bcerende <b>Kjcerlighed</b> vil Dn drage os stedse ncermere til Dig . O Herre , giv os Naade til at lade os drage , til med hver Dag at blive stedse ncermere forenede med Dig . Giv ethvert trolovet Par , at de Begge maae trolove sig stedse inderligere med Dig , blive uadstillelige fra Dig og saaledes ogsaa vcrre indbyrdes forbundne i een og samme Tro , een og samme Frelser , een og samme Retfcerdighed og HelliggMelse og een og samme Vandring til det Jerusalem heroventil . Ak , Herre , lad din Forbarmelse og Salighedeu i Dig kaste sit <b>Blik</b> , sit Morgenfljcer i enhver Forlovelsesstand, for at det ogsaa kan skinne gjennem 2Egtestanden og ovklare Huset tilligemed dets Glceder , dets Mmsommeligheder og Trcrngsler . Det forunde Du af Naade vore forlovede Par ! Amen . He < it cerer Guds dyrebare Ord Forlovelsesstanden . I vor Text sammenligner Gud sin Glcede over sit troe udvalgte Folk med en Brudgoms Glcede over sin Brud . Vor Herre og Frelser kalder sig <b>selv</b> Brudgommen og Menigheden Bruden . Johannes den Dober er Brudgommens Talsmand og Brudens Forer . De ved hans Prcediken ovvakte og i Jordan dsbte Sjele forer han til Herren , Brudgommen. Herren sammenligner de Dage , Han tilbringer her paa Jorden , med den Tid , Brudgommen er hos Brnlluvsgjcesterne. Brylluvsgjcesterne skulle Me sl » rge , saalcrnge Brudgommen er hos dem . Ja , hele den Tid , hvori Kirken er stridende , hvori den forventer Herrens Gjemkomst for at suldende sit Frelservcrrk , sammenlignes med en Forlovelsesstand.

<h4>1024</h4>Ved Vryllupsmaaltidet maa Bordbonnen ikke mangle . Har den end lcenge sovet i en Familie , saa er dog dette en Dag , paa hvilken den bor vcekkes . Og har man aldrig bedet i en Familie , saa maa man dog paa en saadan Dag fole , hvor fattig , hvor tom , hvor told man scetter sig tilbords, nåar man ikle har takket Herren og ilke anraabt Ham om hans Naade i Fremtiden . Endnu paaligger det ester almindelig Vedtcegt Brudens Fader at forrette Bordbonnen paa denne Dag . - Hans Born og Gjcester spise endnu ved hans Bord . Forssmmes Bonnen paa denne Dag , — flutter ilke Pigens Liv dermed i Foreldrenes Huus , saa vil vel heller ikte Livet i hendes eget Huus blive begyndt dermed . Man undlader da at medgive Nornene en af de kosteligfte Bygningsstene i Familielivet . Ved Vryllupsmaaltidet er det en stor Glcede at see begge det unge Pars Familier sammensmeltede til Eet i Herren . I Midten sidder det unge ZEgtepar , og til Hoire og Venstre Fcedrene og Modrene . O , hvor ncer ere ikle disse nu bragte hinanden ! Deres <b>Kjcerlighed</b> og VMner hvile jo nu paa Bornenes fcelles Huus . Og efterdi Mand og <b>Kone</b> ere Eet , bor det unge Par ogsaa staae i et <b>sandt</b> Barneforhold til hinandens Forceldre . Gid enhver ung Mand og ung <b>Kone</b> ret as Hjertet vilde kalde sin 2 Egtefcelles Forceldre Fader og Moder! Gid Forceldrene ogsaa vilde betragte sit nye Barn med ret faderlige og moderlige Folelser ! Det gjor en <b>Hustru</b> ondt , nåar hun vel har vundet sin Mands , men ikke tillige hans Forceldres <b>Hjerte</b> . Da blceser det hende saa koldt imode i hans Familie ; hun forekommer sig <b>selv</b> som en Kvist , der ikke er bleven rigtig indpodet . Sorg og Suk gaaer da allerede tidligt gjennem hendes <b>Hjerte</b> . Herren give , at en oprigtig Forening mellem begge Familier allevegne maa finde Sted ! Dog nok herom . Vi maae endnu omtale nogle Andre , som ogsaa have sine Fordringer til Brylluppet . Overalt i Christenheden , hvor man har feiret en rigtig Glcedesfcest , har man ogsaa betcenkt de Fattige . Glceden udvider Hjertet . Glceden ihukommer ogsaa dem , som Me glcede sig paa den Dag . Gud Fader har indbudt Fattige , Halte , Kroblinger og Blinde til sin Sons Bryllup . Vi ere <b>selv</b> saadanne Gjcester . Saa ville vi da ikke glemme de Fattige i vor Glcede . Paa Landet stal der tidligere have vceret Skik , at man hensatte et eget Fad , hvorpaa man lagde Noget af hver Ret til dem . Dette kaldte man Guds Deel , den Deel , Han ved de Fattige afhentede til sig . Dette er vel nu gaaet af Brug ; men man kan dog endnu give Gud Hans Deel i Penge og andre Gåver . Ogsaa de Fattige stulle bede for det unge Par og takke Gud for

<h4>1068</h4>noermere end en Vens med sin Ven , en Broders med sin Broder , et Barns med sine Forceldre ! Ak , Herre , Herre ! Wgtefolk stulle vcere Eet , stulle vcrre eet Kjod . O , gjor Du ret dem euige ! Bed Du for ethvert christeligt Wgtevar . Lad den Hellig-Aand arbeide paa dem for at drcebe veres Selvraadighed og Egensindighed , deres Hjerters Kulde og Hovmod . Drag dem Begge ind i dit Ord og din Aano . Lad det Ord fra Alteret : „ Hvad Gud har sammenfoiet , stal intet Menneske adstille , " stedse mere og mere blive <b>Sandhed</b> hos den . Herre ! hellerikke den gamle Adam med sin Egenvillie og Trodsighed bor adstille dem . Hellerikke i Hjerterne bor det komme til nogen Adstillelfe og Kulde . Giv tvertimod Naade til at ethvert Par man blive stedse fastere sammenfoiet i Dig , maa faae Dig og hinanden stedse kjoerere og hverken kunne stilles fra Dig eller hinanden . Giv dem dertil Alvor , Aarvaagenhed , Bon , Forsigtighed , Mmnghed , og Taalmodighed . Lad dit Ord boe i ethvert Huus og Dag for Dag blive kraftigere i Hjerterne . Formedelst din hellige Fodsel , formedelst din Trolovelse med vor Slcegt ville Du stjcrnke os denne Naade ! Amen . Ved Alteret er Wgteparret viet og saaledes ester Guds Orden blevet Eet . Begge Parter have <b>selv</b> valgt hinanden og erklceret , at de elske hinanden , ja elste hinanden hoiere end noget andet Menneske . De have givet hinanden det hellige Loste at ville vandre Pilegrimsveien gjennem dette jordiske Liv i Fcellesstab og dele Sorg og Glcede med hinanden. - De boe i Huus sammen ; de stane i et inderligere og fortroligere Forhold til hinanden indbyrdes end til andre Mennester . Og dog er der mangfoldige Par , hos hvilke der aldrig kommer til nogen <b>sand</b> Enighed . Vi ville nu siet ikke tale om de ZEgtefolk , som kun en kold Beregning har fort fammen , i hvilket Tilfcelde det faaledes egentlig ikle er Personerne , men deres Midler , som ere blevne sammenviede. Men <b>selv</b> mellem dem , som have rakt hinanden Haanden as eget Valg og naturlig <b>Kjcerlighed</b> , herster der ofte ingen Eendrcegtighed . Den <b>Kjcerlighed</b> mellem Forlovede, som svcevede i Skyerne , den drommende og svcermeriste <b>Kjcerlighed</b> , der heller vilde doe end give Slip paa den Elflede , hvor snart kjolner Me ofte den ! Mikal , Sauls anden Datter , havde elstet David . Den unge Helt var en Mcmd efter hendes <b>Hjerte</b> . Hun cendsede ikke sin Faders Vre.de ; hun blev Davids <b>Hustru</b> . Da Saul vilde drcebe ham , frelste hun hans Liv . Da Morderne trcengte ind i Huset , havde hun allerede hjulvet David til at stygte . Ved at lede i Sengen fandt Fienderne et Afgudsbillede , fom Mikal havde lagt der istedetfor ham . Da David omsider var

<h4>1100</h4>og Hjcelp . Hvor To ville herske , er der bestandig Ufred ; et godt Raad , en venlig Hjcelp og Tjeneste er derimod stedse velkommen . Derhos bor enhver Svigermoder vel betcenke , at Herren saa at sige nu giver hende Afsted og scetter hende ud af Virksomhed , for at hun , endnu forinden hendes sidste Time kommer , kan indgaae til Guds Hvile . Salig er den , hvem Gud efter Dagens Sved og Moie stjcenker en rolig og stjon Aften , hvori han kan overstue sin Fortid , soge Naade for sine Synder , prise Gud og forberede sig til den Nat , da Ingen kan arbeide . Fra Mandens <b>Moder</b> komme vi til Konens Forceldre . Med Formaning til hjertelig Idmyghed bor de udsende sin Datter til hendes nye Huus . De bor ikke forlange , at Alting der stal vcere ordnet og indrettet som i deres eget Huus . Det er Mandens Huus . Den unge <b>Kone</b> bor anerkjende og agte den Slik , som dersteds herster , forsaavidt som den ikke er syndig . Hun er en Kvist , som er afstaaren af deres Huus ' s Stamme og indpodet i en anden . Podekvisten kan ikke medbringe den gamle Stammes Rodder ; den bliver sammenfoiet med en ny og sat i Forbindelse med dens Rodder . Og fremfor Alt bor den unge Kones Forceldre vogte sig for at ville holde hendes <b>Hjerte</b> tilbage i sit eget Huus og scette hende i en los Stilling , hvori hun ligesom svcever midt imellem begge Familier . Hun har jo hengivet sig ganske til sin Mand , og hendes Forceldre have <b>selv</b> bortgivet hende til ham . Der gives intet Huus , som ikke har sine Sorger og Gjenvordigheder . Ogsaa til dem er hun viet . Forcelorenes Huus bor ikke vcere et Sted , hvor hun kan gaae hen og udose sine Klager , og hvorfra hun vender tilbage til sit eget Huus som fra sit rette Hjem . Se til de Gudfrygtige i den gamle Pagt , ja endog for den gamle Pagt — hvor straaler de ikke ved Siden af mange Christne i vor Tid ! Vethuel og hans Born betragte Rebekka som fuldstcendigt stilt fra deres Stamme ved sit Giftermaaal med Isak . Ved Afsteden sige de ikke et Ord om at hun stal kunne soge nogen Tilflugt i sine Forceldres Huus . Hun maa tåge sit hele <b>Hjerte</b> med sig til det fremmede Land og sit nye Hjem . Den unge Tobias ' s Svigerforeldre , Raguel og hans <b>Hustru</b> Edna , formane sin Datter ved Afsteden til at cere sin Mands Forceldre som sine egne . Ikkedestomindre kan og stal den unge <b>Kone</b> fremdeles bevare sin <b>Kjcerlighed</b> og LErefrygt for sine Forceldre . Ja , dette vil i saa meget hoiere Grad vcere Tilfceldet , nåar de ved sin Viisdom og Erfaring bidrage til at berede hende Sted i hendes nye Hjem . Hun er nu forst og fremst sin Mands <b>Hustru</b> og saa derncest deres Barn . For det Tredie bor Manden med <b>sand</b> Klogstab

<h4>1163</h4>mine gamle Forceldre ; jeg er deres Forsorg erinde , deres StMe ; Gud har sat mig i dette dyrebare Kald , " sua er dette en Grund , som har en god Klang i Guds Uren . Det hcender ogsaa , at en Pige , som Herren ved Dpden har frataget hendes Forlovede , forbliver denne tro i sit hele Liv og aldrig vil indgaae nogen ny Forbindelse . Og endelig ftndes der , stjondt ikte ofte , Saadanne , som troe at kunne tjene Herren bedre i sit Kald som ugifte end som gifte . Ligesom enhver Stand , saaledes har nu ogsaa det ugifte Liv sine Farer . Vi ville her ganske scette dem ud af Betragtning, som forblive eenlige for ikke at opgive sin kjedelige Frihed og sin Bekvemmlighed . Men vi ville navnlig minde vore Loesere om hvor ofte ugifte Personer lige til en W Alder vedblive at elske Fjas og Forfcengelighed . Andre forhcerde sig stedse mere i sin Selvraadighed og Egensindighed; deres sceregne syndige Natur blive ikle bpiet ved en daglig Opofrelfe for <b>Hustru</b> eller Maud , men antager en starp , udprceget Skikkelse . Have de intet bestemt Kald , saa udfylde de ofte sin Tid med Sladder og andre Tungesynder . Hertil kommer Klager over deres eensomme Stilling , Klager over enhver liden Gjenvordighed ; Gud har behandle : dem uretfcerdigt ; de fee med Bitterhed vaa sine Ssstende , der leve i et lykkeligt Familieliv ; de ere starve i sine Domme og blive baade andre og sig <b>selv</b> til Byrde . Deres hele Tilvcerelse dreier sig om dem <b>selv</b> . Hvor vidt forskjellige saadanne Mennester ere fra de Ugifte , fom Apostelen har for <b>Die</b> i vor Text , behove vi ikke ncermere at vaavege . Hvad Raad er der nu herfor ? Hvorledes stulle de kunne bevare sig felv fra de ncevnte Forvildelfer ? Hervaa give vi det tredobbelte Svar : „ Derved , at de grilie Herre » » ' i en barnlig. Tro , vlrlge sig et bestemt Kald og flutte sig til en Familie." Ved den daglige Omgang med Herren holde ogsaa de Eenliges Hjerter sig frists , glade , deeltagende og tilgjengelige . Ved den modarbeides Selvraadigheden , Egensindigheden og Forstenelsen i egne Tanker , Synsmaader og Luner . Ved den holdes <b>Kjcerligheden</b> ilive , saa at man beder for Andre , meddeler Andre og virker for Andre . Vi have kjendt eenlige Perfoner , der i den stille Omgang med sin dyrebare Sjeleven bejad en Rigdom paa <b>Kjcerlighed</b> , Venlighed , Idmyghed og Taalmodighed , som man ikke stulde have ventet at ftnde i deres übekjendte Vråa . Jesus Christus er den , som allevegne beseirer vort <b>Hjertes</b> Forvendthed, Trodsighed , Haardhed og Bitterhed og gyder himmelsk Venlighed og ovofrende <b>Kjcerlighed</b> deri . Hvo som omgaaes med Ham , bestoder den rette Ven , Broder , Mand , Kjcep og Stav . — Vi gaa nu over til Virkekredsen og Familien . Den

<h4>1542</h4>gaaer den Formaning : „ Vcerer gudfrygtige ; studerer flittigt , og beder for os herhenne ! " I den samme Tone strev deres Fader til dem og sendte dem blandt-Andet Luthers Katekisme i det hebraiske , grceste , latinske og tydste Sprog . Hendes Dnste , at det maatte forundes hende , endnu engcing at vederkvcege sig med sin Mand — men hellerikke uden ham — i Friheden og Hjemmet , gik Me i Opfyldelse . Hun dode i Fangenstab 1594. Hendes sidste Bon til de om hendes Seng forsamlede Tjenere gik ud paa , at de ester hendes Dod vilde behandle hendes Mand med fordobletTrostab og Omhyggelighed og vcere ganske det samme for ham , som hun <b>selv</b> saa gjerne havde villei vcere . Dette var en fyrstelig Korsdragerinde . Nu stulle vi fortcelle om en as borgerlig Stand . I en By i det jydlige Tydstland levede der en Borger , som havde faaet Vertshuset kjcerere end sit eget Huus . Aftenerne , der hos os ere ligesom stabte og dannede for Familielivet , tilbragte han udenfor sit Hjem i en Kreds as ryggeslsse og lystige Kammerater . Dette fandt hans <b>Kone</b> sig i med stille Sorg , uden Trcette og stoiende Optrin . Naar han kom hjem , var hun ligefuldt venlig imod ham . Hertil bad hun daglig Herren om Styrke og ncerede detHaab , at hun paa den Maade snarest skulde drage sin Mands <b>Hjerte</b> fra den steile Afgrund op paa den hellige Klippe . Engang , da han seent paa Natten var sammen med sine Svirebrodre , kom Talen paa deres Koner . Han roste sin Hustrues Sagtmodighed og paaltod , at hvis de nu ved Midnat gik hjem til ham og forlangte , at hun skulde lave Mad , dcekke Bordet , og beverte dem , saa vilde hun gjore det uden Knur . De Andre loe ; men han blev ved Sit . Det kom til et Vceddemaal mellem dem og derpaa til et Forsog . Den trofaste <b>Hustru</b> stod op , lavede Mad og opvartede dem ved Bordet , just ikke med Glcede , men dog med Vcerdighed og sommelig Anstand . Da blev Selstabet ilde ved ; Maaltidet varede ikke lcenge ; de Fremmede gik , og Manden blev alene tilbage . Slagen as sin Kones tjenende <b>Kjcerlighed</b> , spurgte han hende hvor hun stk Kraft fra til faadan Troskab . Hun svarede ham : „ Dit Liv er et Liv i Verden ; Verden forgaaer , og dens Lyst ; men bag Verden ligger der en usalig Evighed for alle haardnakkede Foragtere af Gud , folgelig ogsaa for dig , saaledes som du hidtil har levet . Med den Elendighed for <b>Die</b> , som du ganer imode , soler jeg Trang til at gjore dig de faae Aar , som du endnu har tilbage at leve i her paa Jorden eller fom jeg endnu stal leve med dig , fan venlige og behagelige fom muligt . Dette er Grunden til at jeg foier dig i Alt , og Herren giver mig Kraft dertil . " Dette gjorde Udstaget . Dette var den sidste Aften , Manden havde

<h4>1645</h4>Naar du gjsr dig noget mere fortrolig med denne herlige <b>Sandhed</b> , faa staner du alle Born i din Menighed ncer med dit <b>Hjerte</b> . Du maa bede for dem , vaage over dem og rcekke dem en hjcelpfom Hacmd , hvor du kan . De paa sin Side staae da ogfaa dig ncer . Hvo fom impdekommer dem med et faderligt og moderligt <b>Hjerte</b> , for den have da ogfaa de et barnligt <b>Hjerte</b> . Nu kan vistnok Ingen rcekke Alle Haanden vaa famme Maade . Men i enhver Menighed gives der Huller , fom den hjcelpende <b>Kjcerlighed</b> bsr tilstoppe . I enhver Menighed gives der Norn , fom have mister den ene af sine Forceldre eller dem begge . Hvis <b>Kjcerlighed</b> havde disse ifcer Krav paa ? Paaderes , fom Gud ikle har stjcenket Born . I det ene Huus er der ingen Bsrn , i det cmdet ingen Forceldre . Hvad er da klarere end at Gud Herren vil udfylde Mangelen af den ene Deel med den anden ? Men har en Familie med Trostab tåget sig af forceldrelsfe Born , faa behover den vist Me i Alderdommen at klage over Eenfomhed og Tomhed . Erfaringen viser noksom, at der ikle er taknemmeligere Mennester til end forceldrelpse Bern , fom have gjenfundet Fader og <b>Moder</b> . Men ville I virkelig tåge et fremmed Barn til Eder fom jert eget , faa tåger det i faa ung en Alder fom muligt . Dets Oporagelfe vil da falde Eder langt lettere ; det flutter ftg da langt inderligere til Eder , og I blive virkelig Fader og <b>Moder</b> for det . Halvvoxne Born blive ofte bestandig ! fremmede i Hufet , om mcm end behandler dem med aldrig faa megen <b>Kjcerlighed</b> og Trostab . Vel pleie ZEgtefolk felv efter at vcere komne temmelig langt uo i Aarene fremdeles at leve i det Haab , at Gud nok endnu vil velsigne dem med Afkom , og de ovscette derfor fra Tid til anden et faadant Skridt . Deres Haab kan vistnok gaae i Opfyldelse ; men Gud vil dog fnarest vise Barmhjertighed mod dem , som felv have viist Barmhjertighed . For ikte lcenge siden hsrte jeg en gammel barnlps <b>Kone</b> beklage sig og sige : „ leg og min salig Mand have begaaet en stor Forspmmelse . Vi burde allerede for femogtyve Aar siden have tåget en stakkels forceldrelsst Varn til os . Da vi indsaae vor Feil , var det for feent . Da var vi for gamle til at opdrage et lidet Barn , og havde vi tåget et storre til os , vilde det strax have gjort sig Tanker om det lille vi eiede ; det kunde let have eistet vor Formue hoiere end os . Jeg mcerker godt , at alle de , som i fenere Tider have villet flutte sig til os , have vceret ledede af et eller ander Bihenfvn , og jeg foler ret tungt , hvor eenfom jeg staaer <b>selv</b> i min ncermeste Kreds . " Her kan man med <b>Sandhed</b> sige : „ Hvad du gjl » r , det gjsr snart ! " Og da give Naadens Gud dig den famme Glcede af et faadant

<h4>1660</h4>at komme til en <b>sand</b> Omvendelse , det er Noget , som scettes ud af Betragtning . Man sorger og er utrostelig , nåar et Barn nedsynker saa dybt i Uredelighed , Tyveri , Bedrageri , Falstneri og Deslige , at den verdslige Dvrighed maa udstrcekke sin Arm ester det og give det dets velfortjente Lon ien Strafanstlllt . Men <b>selv</b> da er det dog ikke saa meget Synden som Skammen , ikke saameget det forstyrrede indre Liv som den forstyrrede jordifle Lobebane , der bringer Sorg over Sjelen . Noget af det , der volder en Familie den storste Hjertefora . , er et Barns uheldige Giftermaal , nåar Parret ikle kommer godt overeens , nåar den unge Kones Vei gaaer igjennem Graadsdalen , nåar begge 3Egtefcrller maae kjcempe med Trang og Fattigdom , osv. Hvor mange Sukke og Klager over en faadan Lod maa ikke en Geistlig hore af nedboiede Moore ! Men hvor fjelden klage de ikke over Aarfagen til denne Elendighed ! Hvor fjelden sukker man ikke over den dybeste Nod i Huset , nemlig over at Guds Ords Lys aldrig er blevet tcrndt og aldrig har brcendt der ! Endelig ville vi ogsaa omtale den meest bagvendte Sorg , der kan tcenkes i Verden . Endnu i vore Dage gives der Fcedre og Moore , som ikke kunne forsone sig med Tanken om at deres Son eller Datter er kommen til en levende Tro . Naar de ville udtrykke sig rigtig mildt , klage de over at den unge Mand , Pige eller <b>Kone</b> er bleven saa eensidig , over at de negte sig Livets Glceder og give Slip vaa saa mange fordeelagtige Forbindelser . Hervaa have vi ikke et Ord at svare . Saaledes kunne alene de tale , som Intet kjende til Guds Borns Lyksalighed , Intet til den hoieste Forbindelse , der kan gives her i Verden , Intet til Guds naadige Forjcettelfer . Hvor ligger den rette Aarsag til Sorgen ? Hvad sorger vor Herre og Frelser over ? Naar Han udenfor Jerusalem fcelder bitre Taarer over Guds gamle Stad , nåar Hans <b>Hjerte</b> ved den Leilighed bolger og bruser som et dybt oprort Hav , nåar Hans Sjel i Gethsemane er bedrovet indtil Doden , da er det ikke Folkets Armod , som nedboier Ham , da er det ikke Tanken vaa de mange Fattige , Enker , Faderlose , Kroblinger og Elendige i Landet , som nedtrykker Ham , ligesaalidtsom Synet af de romerske Faner , der vaie over Hans Fader Davids Borg ; nei , det et det gamle udvalgte Folks Synder , Vantro , Frafald, der hviler saa tungt vaa Ham og fremkalder Taarerne af Hans <b>Oine</b> ; hvad der smerter ham saa dybt , er ikke en enkelt af Folkets Synder , hvormed det maastee iscer har ftyrtet sig i Ulykke ; nei , det er Summen af Israels Synder , al dets Ugudelighed , hele dets forgelig indvortes Tilstand . Lcerer dog at sorge over Elendighedens Rod ! Hvor Bornene

<h4>2035</h4>Til alle Tider have Mennestene sorget over sine Afdode . Abraham begrceder sin trofaste <b>Hustru</b> , som havde fulgt ham paa hans mmsommelige Vandringer fra Kaldcea til Mesopotamien, til Kana ' ans Land og ZEgypten . Josef begrceder sin gamle Fader , Jakob , og tilligemed ham holde LEgypterne et stort Klagemaal ved hans Grav . Israels Born begrcede Moses i tredive Dage paa Moabs ode Marker . Da den gudfrygtige Kong losias , den sidste vcerdige AEtling as Davrd paa hans Kongethrone , var falden ved Hadadrimmon i Meglddos Dal , holdt Folket en Veklage over ham , som gjenlod lcenge bagefter , idet ingen Konge som han senere opsteg paa hans Fcedres Throne . Gaae vi over til det nye Testament og trcrde ind i den afdode Lazarus ' s eller i lairus ' s Huus eller mode Ligtoget i Ncrrheden af Nam , saa lyde Klagemaal os overalt imode . Og det Samme har vceret Tilfceldet hos alle hedenfle Folkeflag ; <b>selu</b> hos de dybest sunkne af dem findes der Spor af Sorg og Klage over de Afdode . Denne Sorg bor ogsaa vedblive i Christi Kirke . Det maa vcrre nogle letsindige Mennefler , som kunne staae ved sm Faders eller Moders Dodsleie , Ligkiste og Grav uden nogen smertelig Folelse af hvad Gud i dem havde stjcenket dem og af den <b>Kjcerlighed</b> , fom de med dem miste her paa Jorden . Det maa vcere en elendig og letsindig LEgtemcmd , som paa nordamerikansk Viis allerede ved sin forste Hustrues lordfcestelse kan overveie , hvem der stal blive hans anden <b>Kone</b> , , Ved vore Ncerbestcegtedes Dod taler Gud Herren saa traftigt og indtrcengende til os , at denne Tale nok bor gjenlyde en god Tid bagefter og gjore os vort store Tabfoleligt . Vi ville nu for det Forste gjore os <b>selv</b> Reds for hvem vi ssrge over . Vi pleie at sige : „ leg sorger for min Fader, mm <b>Moder</b> , min <b>Kone</b> , min Mand , mit Barn , min Broder , min Soster . " Men dette er tun i visse Tilfcelder <b>sandt</b> . Ere de vandrede bort i Vantro og Ugudelighed , have de lige til sin sidste Stund vceret hildede i Satans Lcenker , da maae vi vistnok sorge for dem ; thi saavidt vi forstaae , ere de da dode en dobbelt Dod . Ak , da er vort <b>Hjerte</b> sonderknust ; da udgyde vi bitre Taarer ; da kan i Grunden heller Ingen troste os . Og Sorgen bliver saa meget dybere, som vi jo ved enhver saadan Leilighed maae sige til os <b>selv</b> : „ Du har Me arbeidet paa denne Sjel med al den christelige <b>Kjcerlighed</b> og Trostab , som du burde . " Denne Sorg ophorer ikke , forend vi have faaet alle vore Synders Forladelse og , fuldendte i Herren , lcere at betragte Historien , » altsaa tillige de Fortabtes Historie , med den guddommelige <b>Sandheds</b> , Retfcerdigheds og Helligheds <b>Oie</b> . Er derimod Nogen hensovet i Herren , saa sorge vi ikke for ham , men

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014031808079'target=_>Besser, W.F., 1862, Johannes's Evangelium</a><h4>2659</h4>til Verden . O Kjcere , fpsrg dig da <b>selv</b> , om den nye Skabning, som alene gjoelder for Gud , i <b>Sandhed</b> er kommen til Liv og Virkelighed i dig ? En <b>Moder</b> kan ikke tage Feil med Hensyn til sit Livs Ssn , om hun har ham , thi med Angest har hun fsdt ham : ligesaa vist vecd den troende Sjel , at den har lesum , thi han er stjoenket den under Fsdselsveer , som et Smertens og Gloedens Barn . „ Den himmelske Fader kalder Ingen Benjamin ( min Hsires Son ) , som ikke Kirken, Moderen , fsrst har kaldet Benoni ( mine Smerters Son ) " ( I . Gerhard ) . Velan da , lad os daglig paany krybe ind i vor Daab , saa stal begge Dele , Moderbcdrovelse og Modcrglcrdc , Bodssmerte og Troesfrndighed , Jesu Dods og hans Livs Samfund vcere vor daglige Christenerfaring . Saadan Erfaring gier os da ogfaa stiklede til som Kirkens Lemmer levende at dele den Bedrsvelse , der i <b>Sandhed</b> stal ende med den Gloede , som den hele Kirke foler , den oedlc Solkvinde , der „ har haarde Fsdselssmerter , " saa ofte en ny Time kommer for hende , i hvilken „ Drengebarnet , som stal regjere alle Folkestag med et lernsviir " bereder sig til at aabenbare sin Herlighed og sit Komme i Skyerne , til at forherlige sit Navn ved sine lcvenoegjsrende og ved sine dsmmende Gjerningcr ( Aab . 12. ) , indtil den sidste Tids ( Matth 24 , 8. ) Fsdselssmerter endelig begynde og Verdens fuldkomne Gjenfsdelse folger derpaa med den fuldkomne Aabenbarclse af den gjenkommende Menncstesons Herlighed , og „ Gud stal aftsrrc alle Taarer af deres Dine " ( Aab . 21,4 . ) , Den herlige Frugt af Discipclbcdrsvelsen bestrider Herren i det trcdeelte Sprog . „ Jeg vil see Eder igjen , og Eders <b>Hjerte</b> skal glcedes , og Ingen tager Eders Gloede fra Eder . " Da Disciplene saae den Opstandne , blev de vistnok glade ( Cap . 20,20 . ) ; dog ved dette Paastegjensyn laa Gjensynet mere paa Herrens , end paa Disciplenes Side ( Jeg , siger han , vil see Eder igjen , ) og der manglede dem endnu Noget i den fulde Gloede , thi endnu betragtede de ham med cengstelige , for hans Forsvinden bekymrede <b>Blik</b> . Forst da de saae ham fare op til Himmelen til sin Fader og deres Fader , bersrte

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014110428001'target=_>, 1851, Menneskets sande Frihed</a><h4>2094</h4>dig vederfaren , at der ikke mere forestaaer dig nogen Kamp , nogen Lidelse og Trcengsel , men at du stal have lutter Fred og gode Dage herneden , da har du intet Begreb om Syn ? dens frygtelige Magt . Vel har den Herre Jesus nu knust Slangens Hoved i dit <b>Hjerte</b> og derved givet den sit Dsdssaar; men endda lever den dog efter sin Slangenatur , og , dersom du ikte ved den Helligaanos Kraft mere og mere dsder den , stal den komme til Krcefter igjen , dens Dsossaar stal lceges , og den stal blive endda meget grummere end for . Den hellige Skrift lcerer os , mine dyrebare Brsdre og Ssstre , at enhver <b>sand</b> Jesu Kristi Discipel har en svcer og alvorlig Kamp at bestaae , saalcenge han endnu vandrer herneden , en Kamp saavel med Synden i sig , som uden , for sig , en Kamp med sit eget syndige Kjsd , som han endnu maa bcere paa , med den onde og forvendte Verden , som omgiver ham , og med Djcevelen , som lurer paa ham som Rovdyret paa sit Bytte . Vorder hellige , thi jeg er hellig , tilraaber Herren hvert Lem af sin Menighed . Paa Netfcerdiggjsrelsen maa Helliggjsrelsen fslge . Den , som har faaet Syndernes Forladelse , synde ikke mere ! Det er Herrens Fordring til Sine . Og det Menneste , som er aftoet fra sine Synder med Forsoningens Blod , et saadant Menneste , i hvis <b>Hjerte</b> Guds <b>Kjcerlighed</b> er udost ved den Helligaand , kan ingen inderligere Lcengsel have end at opfylde denne Fordring . Men dette kan kun stee paa <b>Selv</b> forn « g teisens Vei . Vil Nogen komme efter mig , han forn Eg te sig <b>selv</b> og tage sit Kors paa og fslge mig , siger Herren . Men at forncegte sig <b>selv</b> , det er en usigelig tung og smertefuld Gjerning ; thi hvad er det andet end at opgive alt det , hvortil det naturlige Menneste har meest Lyst , alt det , som er det kjcerest ! hvad er det andet end at rive ud af sit <b>Hjerte</b> enhver Afgud , hvortil det af Naturen har Lyst , hvormeget end Saaret maatte blsde derved ! Hvad er det andet end , som vor Frelser siger , at rive det <b>Die</b> ud , som frister os til Synd , og hugge den Haand af og kaste den bort , som er os til Forargelse ! Derfor bruger Skriften et endnu stcerkere Udtryk , naar den taler om Bekjcempelsen af den Synd , som er i os , det er i vortKjsd . Den siger : dsder Eders jordiske Lemmer! Dersom I dsde Legemets Gjerninger formedelst

<h4>2248</h4>nåar de hore den Syndsbekendelse , dn aftcegger for Gud ; de skulle lcere , hvilke Velgjerninger de tunne udbede sig , nåar de fornemme , hvortil dine Bonner sigte . Dine For ? bonner kunne indgyde i deres Hjerter <b>Kjcerlighed</b> ttl deres Medmennesker , Hengivenhed for den Kirke og det Fcedre ? land , de tilhore , og , hvad vi tillige bsr ansee for meget vigtigt : Folelser af undersaatlig Omhu for Kongen og Kongehuset , nåar de hsre dig at nedkalde Guds Velsig ? nelser over dem ; og dine hoitidelige Taksigelser for Her ? rens Velgjerninger i aandelig og legemlig Henseende stulle opvcetke saadanne Forestillinger hos dem om den forbar ? melsesrige Frelser , at derved , i deres unge Sjiele , kunde blive antiendt Gjenkjcerlighed til Ham , fra hvem alle gode og fuldkomne Gaver tomme . Saaledes stulle de lldt efter lidt og uformcrrkt blive sande Christne , og voxe l Vllsdom , Alder og Naade hos Gud og Mennesker . Ja , dersom Udovelsen af den her fremstillede hellige Pllgt ret over ? holdtes , stulde man med god Grund kunne haabe , at alle christelige Huse , i Forhold til deres Storrelse og Stllltng , stulde blive Planteskoler for <b>sand</b> Gudsfrygt , og du stulle paa den meest overbevisende Maade erkjende Guds Hor ? syns Viisdom , der gjorde Mennestet i dets Barndom saa meget afhcengigt af Forceldrenes Omsorg og saa langt mere uformuende til at sorge for sig <b>selv</b> , end de lavere Skabningers Affodninger . . „ . - . Lad mig da hjertelig bede dtg , hcere Ven , at taste et nyt <b>Blik</b> af Dmhed paa dine Born , nåar de nu atter samles omkring dig ; og sporg dit eget <b>Hjerte</b> , hvorledes du kan forsvare for Gud og for dem , at du derover eller lader dem savne saadanne Formaninger , som de , om hvllke du nu bliver erindret ? Formaninger , uden hvllke det staaer Fare for , at Alt andet , som du kan gjore for dem , bliver til liden eller ingen Nytte , om de end bl eve de lykkeligste i verdslig Henseende ? Thi hvad er verdslig Lykke , uden Kundskab om , Frygt for og Kjcrrllghed tll Gud ? Hvad andet end Sjiele forgift , som opblceser og forvirrer dem ? Hvad andet end et Middel til at gjore dem endnu sikkrere og endnu mere ulykkelige ? Kort sagt , for ikke at tale om den velgjorende Indflydelse , som Gud ? frygtigheds øvelse kan have paa deres timelige Stlllmg , idet den , samler Guds Velsignelser over dem ; thl Gud ? flygtighed er nyttig til alle Ting . 1 Tim . 4 , 8.

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015060948129'target=_>Linderot, Lars, 1847, Lars Linderots Prædikener paa alle Søn- og Helligdage</a><h4>3154</h4>skav . Den , som troer allerede at have naaet saavidt i det Gode og tiltroer sig Fortrin , som han sirtter hsiere end de fortjene , han troer kun altfor ofte , at han ikke behever at arbeide paa sinßedring og kommer stedse derved lcrngere tilbage fra det Gode . Hovmod ' over egen Fortjeneste foder ogsaa Foragt for Andre , som ofte ere bedre Mennester end han , som ncdftetter dem . De forblende ofte sine Dnrkere i den Grad , at de ikke ste det Gode hos Andre , men holde sig for altfor gode og fornemme til at kunne erkjende andre end egne Fortrin , og hvorvcd de imod sine Medmennester bcgaae en Uretfa > rdighed , som ikke undgaaer Hcrvncn . Og just dette er den Tcenkemaade , som gjor dem afstyelige i Guds <b>Oine</b> ; thi Gud staaer de Hoffa ' rdige imod , men de Vdmyge giver han Naade . For at betrygge Dig , min Christen , imod denne Fare , saa prov Dig omhyggcligt og vaag stedse over Dig , bliv aldrig for sikker paa Pligtens og Dydens Vei , og strt aldrig for stor Tillid til dine egne Krcrftcr . Forelcrg Dig meget mere ofte disse Sporgsmaal : Hvorlangt er jeg kommen i det Gode , i mit <b>Hjertes</b> Forbeoring ? Har jeg da virkelig forbedret mine etkjendte , mine angrede Feil ? Har jeg af <b>Kjcerlighed</b> og Taknemlighed imod min Skader gjort det Gode , af <b>sand</b> Tilboielighed og med et reent Forscet ? Eller ovfylder jeg mine Pligter alene af Vane , af Egennytte , af Begjcrrlighed efter 2 Cre og Bifald ? Har jeg tillige udovet Forsonlighedcns , den fredsommelige Kjcrrligheds , Fsielighedens , den velgjorende Medlidenheds Pligter ? Eller vare mine Bestra ' belser blot rcttcde paa de lettere Pligter , som kostede mig mindre Moie ? Den , der paa denne Maade prover sig <b>selv</b> , o , den stal snart stode imod tusinde Mangler og Ufuldkommenheder , som ydmyge ham for Gud og tvinge ham til den Bekjendelse : Herre , hvo kan mcerke , hvor ofte man feiler ? Forlad vore hemmelige Mangler .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111103001'target=_>, 1856, Christiania-Postens Feuilleton</a><h4>2904</h4>Da jeg nceste Dag kom tilbage fra Potsdam , leverede man mig et Brev , det var fra Julie . Jeg aabnedc det paa Trappen og laste : « Har De nogensinde holdt af mig , saa sog at forhindre denne Duel . Jeg har Overbevisningen om , at De kan forhindre den , dersom De vil . Sog at forlige Dem , om det saa stal vere ved at anklage Dem <b>selv</b> . De veed om jeg har eistet Dem , De veed , at vi begge kunne skrifte vor <b>Kjarlighed</b> for Himmelen og at jeg kun frygter Menneskenes smudsige Indbildningskraft , som soger ester det Onde og ester Skandale . Ingen vil troe <b>Sandheden</b> , og destoverre <b>Sandheden</b> er jo paa en vis Maade tilstrekkelig for at nedsette mig i Verdens og i min Mands <b>Oine</b> . Tilgiv , min Ven ! tilgiv ! jeg har maastee varet haard og urctfcndig mod Dem , men De vil domme mig med den Godhet » , jeg har lårt at lase i Deres nfordarvede <b>Hjerte</b> — ak ! jeg Tor af Angst . — Var god og barmhjertig mod mig ; jeg stal aldrig glemme Dem det . Julie E . "

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006081800029'target=_>Wexels, W.A., 1860, Vidnesbyrd fra Kirken</a><h4>523</h4>Og er det da nu ikke inderlig hyggeligt , kjerre chrisine Venner ! at vi i denne Aftenstund ere samlede med hverandre her i Herrens Huus ? er ikke det et godt Tegn paa en velsignet Aftentime , en velsignet Solnedgang , en liflig Aftenrode med velsignet Afsked fra det gamle , det svindende Aar , og saaledes tillige et godt Varsel for den Dag imorgen , for det nye Aar , som dermed ovrinder ? Jo , jeg veed det , Venner . ' at saamange af Gder , som elske denne Herrens Helligdom , dette rolige Paulun , hvor de Storme stilles , som bruse derudenfor , hvor Livet med dets Olcrder , Sorger og Gjerninger forklares , hvor Herren aabenbarer sin Herlighet » , hvor hans Ord bringer Kys og Fred og Styrke , hvor Himmelens , det evige Livs , Krcefter rore sig og de yndigste Udfigter aabnes baade for det smilende og for det taareblamdede Sic , jeg veed det , at I , som saaledes forstaae Davids Ord : " Herre , hvor elskelige ere dine Boliger ! " I , som ogsaa i det nu svindende Aar her fandt det Bedste , I ssgte , I vil ogsaa nu sige : « Her er godt at virre ; her er godt at sige Farvel til det Aar , vi nu atter har oplevet med hverandre . " la , i <b>Sandhed</b> , med hverandre ; thi <b>selv</b> uden det personlige Bekjendtskab er jo vort Menneskeliv et Samliv , eet Liv , som vi alle dele , hvorfor jo ogsaa <b>Kjcrrlighedens</b> Baand skulde omslynge os Alle ; og sjelden har vi vel saa naturlig Anledning til at fole , hvor ? i < er vi ere forenede , vort Samvandringsskab og vort fcrlles Kaar , vor fcrlles Sogen og Stunden , som netop ved Aarsstiftet, oss synderlig Nytaarsaften . Saa vil vi da takke den gode Gud , fordi han nu samler os her , og slutte os til hverandre, og gjore os det saa godt , som vi kan , med ham for <b>Oie</b> og i Tanke , med hans Ord for Ore og i <b>Hjerte</b> . Det bil vel ikke gaae af uden Vemod , ja maaskee hos mangen En

<h4>1067</h4>hans Helligdom , Ord og Sakramenter , foragtel , forternet og bedrevet , krcenket paa det Allerdybeste Fader og <b>Moder</b> , krcrnket i heieste Maal din Nwste paa Mre , Liv og Gods , ja uden Blussel traadt al <b>Sandhed</b> og Kjirrlighed , al Mrbarhed , Tugtigheb og Ret , under Fod , saa skal alt Tette vcere dig forladt for Christi Pines Skyld , verre udslukket i Forsoningens Naadedyb, som Funken i Havet , og Gud vil ikke see andet hos dig end sin Sens dyre Fortjeneste . — Og du , ChristenM , som bliver lunken i Aarvaagenheden og Bonnen og Kampen , som lader dig friste til , hvad din Samvittighed fordemmer , lader dig friste enten til Lempen efter Verden eller til daarlig aandelig Hovmod , eller til Hykleri for dig <b>selv</b> og Gud og Andre , eller i Modgangs Dage til Mishaab og Utaalmodighed, — Ordet ' , for dig ! for dig ! trcrnger md paa dig , advarende , bedende , formanende , at du stal tcrnke paa din korsfcrstede <b>Kjwrlighed</b> og blues og sige til Satan under enhver Skikkelse , i hvilken han kommer til dig : vig fra mig ! — Og , min Christen ! i den sid ste Kamp , naar Dsdens Skygger indhylle dig , naar <b>Oiet</b> og Hjertet skal briste , naar du stal forlade dette Liv og Alt , hvad Jorden tilherer , naar al Tale om , at " Haabet ikke er forbi , at du maaskee endnu kan leve " , er Daarstabens , Forfcengelighedens og Usandhedens Tale , mens det er det Visse , at nu stal du dcc og kan ligesaa lidt kjeve dit Livs Forlengelse , som du kan gjere en Dag deraf ulevet , da , naar du udstrcrkker det bcrvende , men troende <b>Hjerte</b> mod Guds Naades Favn , da kommer Guds Aand med Ordet : for dig , for dig ! han led og dede for dig ! og ferer det til dit <b>Hjerte</b> , at du skal gribe det , og paa og med det Ord : for mig , for mig ! sedelig indslumre , og gaae retfoerdiggjort md i Guds Hvile , md der , hvor Neveren paa

<h4>1401</h4>— naar Lilien blomstrer i Enge , favrere klcrdt end Salomon . i al sin Herlighet » , i denne listige Tid , da alle dybe Lcrngster i Menneskets <b>Hjerte</b> opvaagne , da <b>Oiet</b> klarer sig , og Alt aander Glcrde , fte , da kommer Pintse til os og siger : alle de Syn og Fornemmelser , som saa fsdelig bevcrge og forlyste Eders Hjerter , de ere kun en Skygge og en sandselig Lignelse af den hellige Vaar , i hvilken jeg fodtes , af den store Time , da Gud lod sin Hellig-Aand neddale for at sammensmelte Mennesket med sig i <b>Kjcerlighed</b> , da den himmelske Lue gad hans Helgene Tunger til at udsige hans store og underlige Ting , da Christi Kirkes Lsvhytte hvcelvedes paa Jorden til Luun og Ly og svalende Skygge for hans Bsrn i Stovet ! " — Saalunde. Venner ! faldt Ordene for et Herrens Vidne paa den hellige Pintsefest for 40 Aar siden , altsaa i Tider , hvor en saadan Rost var sjelden at here og ret klang som et fremmed Tungemaal for en i det Jordiske nedsunken , med Christentro übekjendt og det Hellige med koldt <b>Hjerte</b> eller spottende Lcrber bortstodende Slcrgt . Siden hine Dage , da enhver svag , end sige hsirostet Bekjendelse af vor Herre Jesus som oftest blev nwdt med Latter , er det jo blevet anderledes . Herrens Ord har igjen kundgjort sig som en Magt , der trodser baade Harme og Smiil , og har sundet Indgang hos Mange ; Lamgselen ef ^ ter det evige Liv er vaagnet trindt omkring , og den gamle Christentro bekjendes nu blandt Gamle og Unge som <b>Sandheds</b> Tro til Salighed Og dog , dog staaer saa Meget tilbage , dog er der endnu Kulde og Morke i os og om os , dog er det ofte heelt daarligt med Livet , <b>selv</b> der , hvor ßekjendelsen lyder fra Troendes Mund , ei at tale om Mcrngden , der sover i Synden , eller Hyklerne , der tåge det Hellige forfamgeligt ; og vi har endnu Aarsag til at udbryde med hiint Vidne ,

<h4>2181</h4>Saa ste dog efter , du , som forlyster dig altimellem ved Guds Ord og de Syner , det fremstiller for Betragtningen , du , som bevceges derved , du som maaskee endog beundrer dets dybe <b>Sandheder</b> , dets ypperlige og henrivende Skildringer baade af Gud og Mennesket , ja beundrer dets Virkning paaMenneskene, naar det sinder Indgang , beundrer de Sjele , som ved denne Ordets Virkning ere komne i Besiddelse af et Liv i Gud , af en forbausende Kraft , Reenhed , <b>Kjcrrlighed</b> , Tillit » , Hengivenhed , Taalmodighed og Standhaftighet » , Seier over sig <b>selv</b> og Verden , — ste dog ester , om du ikke i Grunden d . e . i dit <b>Hjertes</b> , dit Vcestns Inderste , i din Villie og dm Samvittighet», er aldeles überert eller i al Fald kun ganske flygtigt berort af Ordet , af den <b>Sandhed</b> , der saaledes stiller sig for dit <b>Oie</b> og griber dig ved sin , du kalde det nu himmelske eller strdelige ( moralske ) Skjenhed ! See til , om ikke denne " skjenne <b>Sandhed</b> " sinder en fuldkommen Modscetning i dit Liv , uden at det falder dig ind , at dette skulde dannes derefter , skulde optage <b>Sandheden</b> i sig og blive et Udtryk deraf , uden at det falder dig ind , at i modsat Fald staaer du som des sterre Logner i dig <b>selv</b> , Logner for Gud . jo mere du giver <b>Sandheden</b> Vidnesbyrd og priser dens Skjonhed , uden at det falder dig ind , at det , som du beundrer og bevoeges af i Tanke og Folelst , skal demme dig til Det » en , og dets Savn i dit eget Indre , i din egen Tilvaerelst , blive dig Kilden til den evige Hunger og Torst , som er eet med den evige Pine ! Ja , du Ordets Horer og dets Villeders Betragter ! prov dig alvorlig , om disse Billeder blive Liv og

<h4>2219</h4>stjonne , hvor det er muligt , at vi arme Syndere skulde faae et evigt Liv , som vi dog terste saa ester , erkjende vor fulde Afmagt og Umyndighed til her at tcrnde et Lys . til her at finde og folge en Vei , som skulde fore os md i Himlens Sale ; ja , da stal og vil vi blive som Born i det Stykke , at vi finde os saa ganske hjcelpelsse i os <b>selv</b> og gjerne troe , at , dersom ikke Gud vidste Raad og vilde aabenbare det , da maatte vi evig gaae iblinde , dersom han ikke vilde lade en übegribelig Naade gaae for Ret , vilde gjere et stort Mirakel , da maatte vi uden Redning vcere fortabte , og da stal vi endelig ogsaa blive som Born i det Stykke , at vi gjerne barnlig og eenfoldig troe og holde for idel <b>Sandhed</b> og forlade os paa Alt , hvad Gud siger til os , om det end overgaaer alle vore Tanker , gjerne troe , at hvad der er underligt for vore <b>Oine</b> , er det ikke for hans , at hvad der er umuligt for os , er det ikke for ham , gjerne troe , at han veed Alt , hvad os er forborgent , at hans Naade og <b>Kjcrrlighed</b> kan finde paa Raad for os og vilde aldrig skuffe os med falskt Haab , men mener i <b>Sandhed</b> Alt , hvad den siger , vil holde Alt , hvad den lover , gjerne troe det og overgive os til ham og gaae md paa hans Villie , hvad saa end vor Forstand og vort <b>Hjerte</b> vil sige derimod .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014031808023'target=_>Luther, Martin, 1861, Dr. Martin Luthers udførlige Forklaring over Pauli Brev til Galaterne</a><h4>132</h4>maa kjcempe med Dsden , naar din Samvittighed ihukommer de begangne Tynder og forfaerdes , og Satan med stor Magt gaaer dig under Vinene og tcenker at overvcelde dig med dine Synders store Byrde ligesom med en Syndflod , at stramme og jage dig fra Christo og tilfidst drive dig til Fortvivlelse ; toenk da paa ( siger jeg ) , at du med modigt <b>Hjerte</b> og sta ' rk Tro kan sige ! Christus , Guds Ssn , er hengivet , ikke for de Helliges Retfoerdighed , men forde arme Synderes Uretfærdighed . Var jeg retfærdig og havde ingen Synd , da behsvede jeg ikke Christus , Midleren, til at forsone mig med Gud . Hvorfor vil du da faa forkjcrt gjsre mig til en Helgen , du forbandede Satan , og fordre idel Retfærdighed af mig , som dog ikke har Andet end kun idel Synder , og det ikke'opdigtede, men ret virkelige ; ikke lette og ringe , men ret store , grove og overmande svaere Synder , saasom stor Vantro , at jeg ikke troer Gud , tvivler paa hans Godhed , daglig foragter ham , er bam fiendsk , aldrig ret erkjender ham , spotter ham , er utaknemlig mod ham , misbruger hans Navn , forsommer hans hellige Ord , bliver kjed og led af det og foragter det osv. , hvilket jo er idel store , grove , svare Synder mod den fsrste Tavle . 79. Desuden er der endnu ogsaa de grove kjsdclige Synder mod storc den anden Tavle , nemlig at man ikke holder sine Foroeldre i Agt og " g ' . " angc 3 Ere , ikke er ivrigheden underdanig , begjoerer en Andens Gods <b>Hustru</b> o , s . v. shvorml saadanne Synder ere hel ringe og übetydelige imod dem , som ste mod den fsrste Tavle ) . Og lad det endog faa mere , Christus at jeg ikke i Gjerningen har fuldbragt Noget af Mord , Hoer , Tyven har hengiog andre deslige grove Synder mod den anden Tavle , faa har jeg dog si « . i Hjertet forsyndet mig derimod og er derfor en Overtroeder af alle Guds Bud ; saamangc ere ( desværre ! ) mine Synder , at de visseligen ikke alle kunde skrives paa en stor Orehud , ja , umuligt er det , at " man stulde kunne tcelle dem , thi de ere talrigere end Havets <b>Sand</b> ( Manasse Bsn V . 91. Desuden er Djcevelcn en saadan Tusindkunstner og behcendig Mester , at han ogsaa kan gjsre mine allerbedste Gjerninger og min Retfærdighed til de allerstsrste Synder . Efterdi det altsaa er saadant Alvor med mine Synder , hvilke ere rette , virkelige , store , grove , gruelige , utallige og uovervindelige Synder , og min egen Retfærdighed slet ikke kan nytte , men kun skade mig , derfor ' er Christus , Guds Ssn , hengivet i Dsden for at udslette dem og gjsre mig og Alle , som tro derpaa , salige ; ham veere Lov og Pris for saadan uudsigelig <b>Kjcrrlighed</b> . Amen . 80. Derfor er det i hsieste Grad magtpaaliggende for vor Salighed, at disse Ord holdes for alvorlige og sande Ord . Og dette siger jeg isandhed ikke omsonst ; thi jeg har ofte erfaret og erfarer endnu daglig, jo loengere jo mere , hvor over al Maade vanskeligt det er at tro , isoer naar den elendige Samvittighed har sin Nsd og sit Svedbad , at Christus er hengiven , ikke for dem , fom ere hellige , retfærdige , ucerdige og hans Venner , men for de Ugudelige , Synderne , de Uocerdige og for dem , som ere hans ssi ender , hvilke have fortjent Guds Vrede , den evige Dsd og Fordsmmelse . 81. Derfor flnlle vi vel ruste og berede vore Hjerter med dette og Sjcelens andre deslige Sprog af St . Paulus , for at vi , naar Djaevelen engang Kiv med kommer og anklager os og siger ! Se , du er en Synder , derfor maa du I " / " " ' blive fordsmt , da kunne msde ham og svare i Ja , kjoere Djævel , netop M derfor , at du anklager og vil fordsmme mig som en Synder , vil jeg Stund , voere retfærdig og from , ikke blive fordsmt , men meget mere salig . Og om han da ogsaa bliver ved og siger : Kort og godt : du maa gaa fortabt, at du da ved at holde dig opreist og staa fast ' og sige : Nei , ikke faa ; A jeg holder mig til Christum , som har hengivet sig <b>selv</b> for mine

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100703005'target=_>Luther, Martin, 1855, Frelsens Olie</a><h4>2221</h4>i denne Anledning , at <b>selv</b> om Himlen stod vid aaben for et Menneske , saa skulde han dog ikke onste sig derind , med mindre det var Guds Villie ;  thi vi stulle ikke bede om andet , end som hvad der er ester Guds Villie . " Men vi svare ganske enfoldigt : Herren taler her med sine Christne , thi de Andre stulle og kunne slet ikke bede ; men de Chnstne vide ikke at udbede sig noget bedre end Guds Navn Rige og Villie , og de kjende ikke til stsrre Fryd end naar denne Bsn opfyldes . Men det er altsammen sagt imod de falske Hyklere, der forvandste Guds Orden og soge kun deres eget , og ville <b>selv</b> voere alt og opnaae Alt bos Gud , og bcere Navn af Christne , stjsndt der hos dem ikke findes nogen <b>sand</b> Glcede over ChristW eller nogen oprigtig <b>Kjcerlighed</b> til Hans Rige ; Bonnen er hul og <b>Kjcerligheden</b> fremhyklet . Vore Skoletheologer , der have en fin Ncese , tale ogsaa om to Slags Kjcerligbed : den lidenstabelige og venskabelige ^ ) , og St . Augustin betegner den fsrste som den der tåger Hensyn til Nytten og Nydelsen ^ ) :  nogle , siger Han , elske Gnd af et oprigtigt <b>Hjcerte</b> , men andre kun for Nyttens Skuld . " Men at den Ene elsker den Anden for at faae fine Lyster styrede , det kalder man Brynde og Boleri , hvorimod den cegte <b>Kjcerlighed</b> er som den virkelige eller naturlige mellem Mand og <b>Hustru</b> Foraldre og Born , hvor den Ene under den Anden det bedste , og scetter sin Lyst og Glcede i hans Held og Lykke . Der er altsaa to Maader at elske Gud paa ; Nogle tcenke , at hvis de blot kunde undgaae Straffen og Pinen , saa bryde de sig ikke det mindsteom hvad der bliver af Guds Navn og Rige . Det er samme fremhyklede <b>Kjcerlighed</b> som naar en Tyv elsker Birkedommeren og Rakkeren , fordi han foler Kildren om Halsen af Frygt for Galgen ; og det gaacr her som man siger , at hvem der er bange for Helvede han er en sikker Dodens Kandidat . Slige Mennesker kjende slet ikke noget til den Glcede som Christus her omtaler . De Christne derimod , om hvem Talen her er og som elske Christum af et oprigtigt <b>Hjcerte</b> , de vide ikke at begjcere eller attraae noget hoiere , end Guds Riges Fremgang , Hans Navns Bekjendelse , Hans Wrcs Bersmmelse og Hans Villies Efterlevelse i den ganste Vlrden; og naar det fsrst ster , da ere de hjulpne i Alt , hvorfor

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081903012'target=_>, 1859, Christelige Psalmer fra ældre og nyere Tid</a><h4>154</h4>7. t . 2 kjerre Sjel , opmuntre dig , Med Andagt at betragte , Hvor Gud har aabenbaret sig , Hvor heit man ham bor agte , At han i Himlens Paradiis Er stor forudenLige , Og bor at have Lov og Priis Paa ganste lorderige Som alle Naaders Kilde . 2. Du , Gud ! est ene Gud , din Priis Du ingen Afgud giver , Kun du er mcegtig , stor og viis , Din AZren ene bliver . Paa dig alene stal vi troe , Til dig i Nsden trcrde , Og ssge al vor Sjelero I din Forenings Glcede Og aldrig fra dig vige . 3. Du est en Aand , o store Gud ! Og Aandens Himmels gaver Til dine Bsrn du deler ud , Som hver det nodig haver . Fra Evighed din Tbrone siaaer , Du har ei Maal og Ende , Og Ingen fra dit <b>Die</b> gaacr , Hvor han sig end vil vende . Du seer hvert <b>Hjertes</b> Tanke . 4. Du en almcegtig Konge er Med Guddoms Kraft og Vcelde , Du hjcelper Den , som har dig kjcer , Og Fienden kan du fcrlde . Din Viisdom derhos overgaaer Al vor Forstand og Evne , Ei nogen Trcengsel er saa haard , Og ingen Nsd at ncevne , Hvor du ei Naad kan finde . 5. Gud , du esi hellig , Uret kan Umulig dig behage , Vil Nogen staae i Naadens Stand , Han Synden stal forsage . Men du er mild og <b>kjcerlig</b> mod Enhver , som Synden hader , I dig han har saa eiegod Og himmelsod en Fader . O maatte vi dig glcrde ! 6. Uryggelig din <b>Sandhed</b> staaer , Vel Den , som det kan kjende ! Al Verdens Trofasihed forgaaer , Men din har ingen Ende . Af ingen Tvang din Villje veed , Du gjsr hvad dig behager , Men altid er du <b>Kjcerlighed</b> I hvad du foretager . At vi paa dig stal bygge . 7. Naar , Gud , jeg dig saaledes seer Som Den , der aldrig sviger , Da glceder det mit <b>Hjerte</b> meer . End alle Verdens Niger , Til jeg ved dig i Himmerig Mig evig stal fornsie . Og uden Ophor gladelig Din Liflighed ophsie Med Engle , Maal og Stemme . 8. O Fader , Ssn og HelligMand , Som vil af Alle prises ! O , los du <b>selv</b> vort Tungebaand , Og lad det klarlig vises . At Du er den , hvis Naade kan Os dertil ret berede , At vi i Verdens vide Land Din TEre glad udbrede , Indtil du hjem os kalder !

<h4>1780</h4>429. . 5. Den Sjel , som Gud i <b>Sandhed</b> kjender , Og sinder Smag i Livets Ord , Som sig med Lyst fra Verden vender Og med sit Sind i Himlen boer : Den kan sig ret i Gud fornoie . Og Verden er den derimod Som Rsgen for det svage Vie ; For den er Gud alene god . 2. Guds Vcrsen den med Lyst betragter , Og paa hans Gjerning glad den seer ; Gud for sin Hjerte-Skat den agter . Den vil og kan ei have Meer , Forsmaaer det Mindste , sonr det Storste Af Det , som Kjedet have vil : Gud er den Sid / ste som den Forste , Hvem al dens Lcrngsel stunder til ' 3. Og . han , som Sjelens Skat vil virre , Han <b>Selv</b> er idel <b>Kjcerlighed</b> ; Det alle stable Ting os ltrre , Ia det er deres Siemed : Vort <b>Hjerte</b> fra dem <b>selv</b> at rukke Og vise det henop til Gud , Som Morkel kan med Lyset smykke Og Alting fik af Intet ud . 4. Og nu dlt Dybe og det Hoie Af <b>Kjcrrlighed</b> : at Gud har vendt Til dem saa naadefuldt et < <b>Oie</b> , Som evig Skjlendsel har fortjent ! — Saaledes soger han at drage Vort <b>Hjerte</b> hen til sig saa nar , At vi i Troens Kraft kan smage , Hvor inderlig han har os kjerr ! 5. Jo mere nu af Aandens Gave En naadehungrig Sjel har smagt , Des Mere onster den at have Og trcrttes ei paa Vonnens Vagt . I <b>Kjcrrlighed</b> at kunne trtrnge Ret ind udi Guds Faderfavn , Og fast ved ham at kunne hcrnge — Det soger den i Jesu Navn ! 6. Dog , her vil Synden tit formorke Den rene Kjcrrligl hedens Brand ; Forst hisset faaer den al sin Styrke , Naar Intet meer den hindre kan . Naar derfor den til Salems Porte Har Tanken vendt og opad seer , Da sukker den : o var det borte , Som trykker os i dette Leer !

<h4>2339</h4>Fader vcere vil ! Hor nu min Von , og lceg dit Amen til ! 2. Du kaldte mig af Naade her til Livet , Og Legem , Sjel og Alt har du mig givet , Ia endnu det , som meget Mere er : I Daaben du mig tvcetted reen og stjcer ! 3. O giv , at jeg min Pagt maa vel bevare , Saa jeg fra dig , min Gud , ei vild stal fare ! Jeg ikke <b>selv</b> min Fod at fore veed : Du , Fader , paa den rette Vei mig led ! 4. Hvad nyttigt er , det lad mig noie lcere , Beflitte mig paa Alt , hvad dig gjor 3 Ere , Ia giv , at i min Ungdomstid jeg maa Dig , o min Skaber , <b>kjcerlig</b> tcenke paa ! 5. Min Fader og min <b>Moder</b> lier mig lyde , Lad <b>Kjoerlighed</b> til dem mit <b>Hjerte</b> pryde , Og hjcelv mig , at mit Liv maa bcere Frugt Af deres Naad , Formaning , Bon og Tugt ! 6. Bevar mig , gode Gud , fra ondt Exempel , At Intet stade stal dit eget Tempel ! Slet Omgang lcrr du mig bestandig fiye Og Logn og onde Ord som Gift at stye ! 7. Din gode Aand lad over Hjertet raade , Saa jeg gaacr frem i Viisdom vg i Naade , Og fslger tro dit Fader-Ord , o Gud , Og har for <b>Die</b> flittig dine Bud ! 8. Mit Liv hernede vil jeg dig befale Og hver en Dag om Bistand dig vaakalde , At jeg indtil min Dod maa blive ved I christen Tro og Haab og <b>Kjcerlighed</b> !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111400044'target=_>James, John Angell, 1878, unge Kvinde</a><h4>292</h4>Eller stulle vi fra denne enkelte Virkekreds kaste vort <b>Blik</b> paa det almindelige gode , som hun ved sin Indflydelse udsver paa Folket ? Hvo ved ikke , at Kilden til et Lands Velfcrrd er at ssge indenfor den huslige Kreds , i lykkelige Familieforhold . Blot en eneste saadan Familie er et Bidrag til dets Magt og Velstand . Kunne saadanne Familier beståa uden <b>Kvindens</b> Rsgt , Omsorg og Ledelse ? Gr det ei blevet et Ordsprog , at Hjemmet altid har vceret Planteskolen for store Mcend , og Msdrene deres Lærerinder ? Man kan vove at paastaa som en almindelig <b>Sandhed</b> , at <b>Kvinden</b> ikke alene er sit Barns legemlige <b>Moder</b> , men ogsaa dets aandelige . Hende er den fsrste Bearbeidelse af dets <b>Hjerte</b> og Sjcel anbetroet . Tanke , Fslelse , Indbildningskraft, Dyd , Religion eller ogsaa Modscetningen dertil , alt voxer under hendes opdragende Indflydelse , Den stsrste moralske Magt er den , en <b>Kvinde</b> udsver over sine Bsrn , Det asgjsrende Dieblik i Opdragelsen er just Begyndelsesvunktet <b>selv</b> . Den herskende Retning , som flat bestemme hele Livets Lov , ligger allerede skjult i de fsrste Barndomsaar ; men disse tilhsre Moderen ,

<h4>335</h4>som <b>Kvindens</b> , intet <b>Hjerte</b> saa omt som hendes . Mandens Bryst kan svulme af Sorg , FrygtMsnderslide hans Sjcel , men scrt ham ved Sygesengen , i Skinnet eller snarere i Styggen af Natlampen , lad ham ensom med Sygdom og Dsd tlrlle Nattens lange Timer og ssvnlss afvente Dagens snigende Straale i Lidelsens Kammer , lad ham kaldes til denne Tjeneste <b>selv</b> for en Broder eller en Fader, og hans stcerkere Natur , <b>selv</b> naar den er mest fuldkommen, vil trcrttes , han <b>Die</b> tillukkes , hans Sind blive utaalmodigt ved denne Ensformighet » , og skjont hans Kjcerlighed, hans Wngstelse er den samme , vil han i sig fole denne uhelbredelige Selviskhet » , for hvilken han vel maa blues , men som trods alle hans Anstrengelser alligevel vil blive tilbage som noget udmcrrkende for hans Natur og et Bevis paa hans Underlegenhed , idetmindste i en Henseende. Se derimod en <b>Moder</b> , en Ssster , en Wgtefcrlle paa denne Plads ; <b>Kvinden</b> foler ingen Trcethed , har ingen Tanke for sig <b>selv</b> . I Taushed , i Nattens Stilhed holder hun ikke blot villig , men ogsaa gjerne ud . Hendes Tre opfatter Lyd , som kun den Blinde eller <b>Kjcrrlighedens</b> Instinkt kan opfatte . Hendes Trin ere lydlose som en Aands ; hvis hun taler , er hendes Stemme mild , og hvor harmonisk lyder den ikke for den Syges Bre , fordi den udtrykker alt , hvad Lyden kan udtrykke af Deltagelse, Trsst og Dmhed ; og saa passer hun ham Nat efter Nat , naar al anden Aarvaagenhed er udmattet . Hendes <b>Die</b> tillukkes ikke , hendes Sind bliver ikke treet , hendes Natur, som ved enhver anden Leilighed er svag , vinder nu en overnaturlig Styrke , hun glemmer sig <b>selv</b> , hun er blot <b>Kvinde</b> .

<h4>499</h4>Og nu ved Afflutningen af dette Kapitel , lader mig opfordre eder , I unge , til allerede ved Livets Begyndelfe at betcenke det store Maal , for hvilket I af Skaberen fattes ind i Verden . Tcrnker ikke altfor hoit om eder <b>selv</b> , thi I ere Syndere , I saavelsom alle . I vehsve , I ogsaa , og I kunne ogsaa vinde den Frelse , som er beredt i Jesus Kristus . Tamker ikke heller altfor ringe om eder ; thi I ere udsdelige Vcesener og kunne arve et evigt Liv , Hcever eder til eders fande Vcerdighed ved at hoeve eder til en <b>sand</b> Gudsfrygt , Tro og et Liv i <b>Sandheden</b> . Forspilder ikke eders Liv i smaaligt , om ogsaa uflyldigt Tidsfordriv , uvcerdigt eders Evner , eders Bestemmelfe og eders Pligt . Betragter med klart <b>Blik</b> og fuld Bevidsthed eders Kald for denne Verden og eders Bestemmelse for den tilkommende . Ser , hvorledes et Kald her aabner sig for eder , hvilket I maa opfylde med en Kristens Hellighet » , Mre og Velsignelse saavelsom med alle en <b>Kvindes</b> Dyder og Fortrin . Drager eders <b>Hjerte</b> fra Forfcengeligheden, stjcrnker det til Gudsfrygten . Skjcenker Gud eders Livs Morgen , og da stal dets Dag , den blive lang eller kort , klar eller overskyet , ensom eller tilbragt i Forening med

<h4>564</h4>Mesters Wre . Maafle ville disse Blade leses af nogen paa samme Maade begavet <b>Kvinde</b> , som har et <b>Hjerte</b> for Kristus og hans Sag , men ringe Midler dertil . Til hende vil jeg sige :  Gak hen og gjor ligesaa " . Der findes jo ogsaa en anden Maade , hvorpaa de unge <b>Kvinder</b> , som ikke ere istand til at gjore meget ved finere eller kunstigere Arbeider , alligevel kunne med stor Velsignelse arbeide for Gud og for sine Medmennesker , uden i den allerringeste Maade at bryde med det passende eller saare Bluferdigheden ; jeg mener den <b>Kjerlighed</b> , som besoger Sygeleiet eller den fattiges Hytte , som til de lidende eller vankundige af sit eget Kjon bringer Naadens Trost og Evangeliets Lys . Sikkerlig er det hverken nogen Indtrcengen i Manoens Rettigheder eller Prestens Pligter , at den gudfrygtige og hellige <b>Kvinde</b> , <b>selv</b> om hun er yngre ( jeg taler naturligvis ikke om Born ) , gaar til en lidende Medkristen og indgyder i hendes Bre de Ord og i hendes <b>Hjerte</b> de hellige <b>Sandheder</b> , som ere den bedste Balsam for en lidende Aand , den eneste Trost for et knust <b>Hjerte</b> . Ikke heller tan det vere utilborligt for hende at tåge Plads ved en fattig Moders Side , som , medens hun er optagen af Naalen eller Vuggen , er rede til at hore Frelsens Ord . Se her er ncesten uopdyrket Arbeidsmark , som aabner sig for eders Flid , for eders Bestrebelser , I unge kristne <b>Kvinder</b> , som have Tid og Anledning til saadanne Beskjeftigelser ! Hvor mange tusinde fattige <b>Kvinder</b> af Arbeidsklassen findes ikke i enhver storre Stad , hvilke ere saa optagne af Husets Forretninger og sine Mcends Fordringer , at de aldrig faa Tid til at besoge den offentlige Gudstjeneste og hore Prekenens og Sangens liflige Lyd , og hvilke skulde hilse som en Fredens Engel den <b>Kvinde</b> , som vilde komme at besoge dem under deres uafbrudte , ensformige Arbeider

<h4>1099</h4>Men er det vel at vente , eller er det endog muligt , at nogen kan undervise med Held i en Religion , som man ei <b>selv</b> kjender og udsver ; kan nogen lede sit Barn til den Frelser , som han <b>selv</b> ei kjender ? Derfor maa en Moders <b>Hjerte</b> fremfor alt verre opfyldt af en <b>sand</b> Gudsfrygt , hvis hun deri stal kunne undervise sit Barn . Kan hun vel uden dette have Lyst til at undervise , <b>Hjerte</b> til at bede . Ret til at haabe ? <b>Moder</b> , kan du tcenke dig noget « Mere , hoiere , vigtigere Kald end at aabne det undrende Barnesind for den forste Tanke om Gud ? At lede dit eget Barn til det Gudebarn , som fsdtes i Bethlehem , var sine Forceldre underdanig , vozte oft til en Frelser , som sagde :  Lader de smaa Bsrn komme til mig , og forhindrer dem ikke ; thi Guds Rige hore saadanne til , " tog dem i Favn og velsignede dem , og som endelig led Korsets Dsd for at give dem et evigt Liv ? Eller at tale for det om Himmelen, hvor Gud og de hellige Engle bo ? O , at se det fsrste , hellige fpsrgende <b>Blik</b> , den fsrste Taare i troende Dine , at hsre det fsrste Spsrgsmaal eller den fsrste Bsn fra spcede Barnelover ! Hvor mangen <b>Kvindes</b> <b>Hjerte</b> har ei ved en saadan Anledning strsmmet over af Glcede og

<h4>1242</h4>dette , til dem snskede jeg at sige : Kaster Mildhedens , Mmyghedens og Blufcrrdighedens Slsr over alle disse skjsnne Former a f Virksomhed og Kraft . Glemmer fremfor alt ikke det elskelige Trcek :  KMigheds Lov er paa hendes Tunge , " Forener <b>kvindelig</b> Amhed og Blidhed med al denne mandige Kraft i Udfsrelse af Livets Pligter . Hvilke eders Egenskaber end maa vcere , vcerer fremfor alt Kvmder ; og naar I , i Egenstab af Wgtefa-lle , <b>Moder</b> og Husbestyrerinde , udove den Magt og Myndighed , som eder tilkommer , saa glemmer dog ikke , at der findes en i Familien , jeg mener eders Mand , hvis Myndighed er endnu stsrre end eders , og at det er paa engang eders Pligt og stal blive eders Lykke med Vnched og <b>Kjerlighed</b> , ei med slavisk Underdanighed , at underkaste eder ham . Unge <b>Kvinder</b> , jeg legger eder paa <b>Hjerte</b> at gjore eder fortrolige med dette sijsnne Sted i den hellige Skrift . Gjorer det til Gjenstand for eders stadige Betragtning ; meget , scerdeles meget kunne I lcere deraf . I ville der se , at Gudsfrygt er den sikreste og den fasteste Grundvold for al <b>kvindelig</b> Dyd , og at den langtfra at opheve eller trede vore jordiske Pligter for ncer , tvertimod overalt , hvor den er <b>sand</b> og levende , skjenker et klarere <b>Blik</b> paa dem og en stsrre Evne til at opfylde dem . Gudsfrygt er nyttig til alle Ting og befordrer alle lovlige Bestrebelser . Der findes ei et eneste cedelt Karaktertrcrk , som ei Religionen endnu mere hcever og foredler , ingen jordisk Interesse , forudfat at den er tilladelig . som den ei merkelig befordrer . Lader ikke dens Fiender indvirke paa eder med sine urigtige Forestillinger , ligesom om Gudsfrygt var uforenelig med almindelig Dyd , stridende mod al personlig Lykke , ligesom om den anbefalede de Pligter , der ere til Hinder for , men forbod de Gleder , som ere nsdvendige for et lykkeligt Ungdomstid . Disse Indvendinger ville ved samvittighedsfuld

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081303006'target=_>Besser, W.F., 1858, Lukas's Evangelium</a><h4>2215</h4>26. 29. ) . Paa det Nsieste bestrider Herren Samaritanerens opofrende , tjenende og oftfindsomme <b>Kjcrrlighed</b> , en <b>Kjcrrlighed</b> , der — som H . Muller siger — har sin Rod i Hjertet , blomstrer paa Tungen og bcrrer Frugt i Gjerningerne , den <b>Kjcrrlighed</b> , som i Gjerning og <b>Sandhed</b> „ elster Nersten som sig <b>selv</b> . " Af Barmhjertighedens Kilde i hans Sjcrl ( „ hcm ynkedes inderligen " ) udvcrldede Forbarmelsens Strsm i <b>Kjcerligheds</b> Gjerninger. I hans <b>Hjerte</b> var det indstrevet , at der til at holde det femte Bud fordres ei alene „ at vi ikke stade vor Ncrstes Legeme eller tilfole ham noget Ondt derpaa , " men tillige , „ at vi hjcrlpe og staae ham bi i al legemlig Nsd . " Vi stulde aldrig lcrse Texten om den barmhjertige Samaritan , nden til vor dybe Bcskjcrmmelse at stille os for <b>Die</b> saadanne Menneskers christelige Gjerninger, som tilhore vildfarende Samfund . „ En Samaritan reiste " — saaledes lyder den ( usynlige ) Overskrift over saa mangen „ Efterretning fra Guds Rige . " — Den Skriftkloge maatte fsle sig meget forlegen og trusten ; han kunde ikke dslge for sig <b>selv</b> , med hvem han vilde have reist , om han havde vcrret En af de Tre . Nu spsrger Herren ham :

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081903009'target=_>Nissen, Johan., 1865, Nissens Samtaler over Luthers lille Katechismus</a><h4>173</h4>ssjendte man paa ham for hans skjendige Handlemaade . Men han sagde : Havde min <b>Moder</b> straffet mig for mine Tyverier og Lsgn medens jeg endnu var liden , saa var jeg nu ikke bleven fsrt til ' Rettclstedet . Hvorledes - opvekker og befordrer en <b>Moder</b> sit Barns Gudsfrygt ? Paa tre Maader : Ved eget Exempel , Undervisning og Vsn . — Eget Exempel . Paulus forlanger nf en <b>Moder</b> , at hun fsrst <b>selv</b> stal blive i Tro , i <b>Kjerlighed</b> , t Helliggjsrelse og i Tugt . Hvorfor forlanger Paulus det ? Det gaar som ' en Digter har sagt : Elster du det <b>Hjerte</b> , som dsende brast for dig , saa vil ogsaa mange andre Hjerter elste ligesom dig , og fsrst og fremmest Barnets <b>Hjerte</b> . Saaledes har det ogsaa veret med de Msdre , som vi ovenfor have omtalt . Hvorledes ? En from Mand sagde : jeg har indsuget Gudsfrygt met » Modermelken . Hvad meuer han ? Thi ogsaa i denne Henseende er det en <b>Sandhed</b> , at Wblet ikke falder langt fra Stammen . Hvorledes stnl Moderen drage Omsorg for , at bendes Bnrn beder ? Hendes Barn stal ikke lyve ' ? ikke bedrage? ikke vere forfengeligt ? Og anfsrer <b>selv</b> mere . — Foreldrenes eget Ercmpel kan ikke erstattes ved noget andet . For Fader og <b>Moder</b> gjelder det , hvad Paulus fordrer af Timotheus 1 Tim . 4 , 14 , at hau stal vere et Exempel , i Omgjeugelse , i Kjerlighed, i Aand , i Tro , i Kydsthed — Undervisning . Men hvorledes kan en <b>Moder</b> uere Lerer . da jo den , som stal vere Lerer , i flere Aar maa uddannes dertil ? Sig nu Eders Mening derom , som I bedst kunne ! Hun kan leilighedsviis og ganske naturlig vise hen til Gud . Hun kan f . Ex . spsrge : Hvem giver os Brsd ? Hvem lader 3 Eblet voxe paa Treet ? Tror du ikke , at Gud ved . at du har sagt Usandhed ? Vil du bedrsve Gud , som er sna god imod dig , ved nt vere uartig ? Ved du ikke , at du engang stnl gjsre Negnstnb for Gud ? Moderen mna anstille gudfrygtige Ovelser med Barnet , saa det at vere gudfrygtig bliver det en Sedvane . Vi ville nevne nogle saadanne Ovelser : Bord-bsn, Morgen- og Aftennndngt , Ssndagcns Hsitideligholdelse , Psalmesaug — og desuden . <b>Sandhed</b> , Opngtighed og Wrlighed i alle Ting , — Velgjerninger og Tilgivelse . Undervisningen maa vere ledsaget af Udsvelse , Ordet af Gjerning , ellers har det ingen Betydning. Se Inc . 2 , 26. — Bsn . Der staar i en gammel Bsn : „ Forud ' en dig o Gud , slet intet vi formåa ; men af din Naadehaand alt Godt vi har og faa " . Men Guds Hjelp maa man bede om . Den fromme <b>Moder</b> vil bede Gud om , nt hnn stal velsigne hendes Opdragelse og al hendes Moie for sit Barn . Det gjorde Monika . Hvad sagde Bistoppen til hende ? Og det stede . Hsrer til Slutning en lideu Fortelling om hvilken Magt en <b>Moder</b> , ja endog det blotte Modernnvn kan have over Barnet . En from Mand gik i en Kirke , hvor man netop skulde holde Gudstjeneste for Ssfolk . Ligeoverfor saa han en aldrende Matros sidde foran Dsren til et Verts-

<h4>481</h4>Mgtestabets Heder . Ebr . 13 , 4 : . Mgtestabet vere bederligt hos Alle ; " det stal holdes i Heder og 3 Ere . Og hvorfor ? Fordi det er en guddommelig Indstiftelse ; den er umiddelbart stiftet af Gud , thi <b>selv</b> fsrte han Eva til Adam . Gud <b>selv</b> er den fsrste Forlover , kan man gjerne sige . „ Du ledte frem , algode Gud , Til fsrste Mand den fsrste Brud Og bsd dem et at vere . " Hoer og Wgtestabsbrud er derfor en Skjendsel og en Vederstyggelighet» for Herren . Og Skjsrlevnere og Hoerkarle stal Gud domme . Ebr . 13,4 . ZEgtestabet er fremdeles den fsrste Stand , som Gud har indsat btandt Menneskene . Mgteskabets Hensigt . Da Gud havde stabt Adam og Eva velsignede han dem og sagde : verer frugtbare og formerer ' Eder . 1 Mosb . 1 , 28 og Psalme 127 , 3 siger : ' se , Vsrn ' ere Herrens Arv , Livsens Frugt en Lsn . Hensigten med LEgtestabct er altsaa 1. Verdens Befolkning og Mennesteslegtens fortsatte Tilvere! se . Men Fader og <b>Moder</b> have ikke opfyldt deres Pligt alene ved at avle Vsrn , men disse Bsrn skulde de opfsde i Tugt og Herrens Formaning ( Eph . 6 , 4 ) ; de stulle vaage over deres Sjele , for hvilke de engang stulle aflegge Regnstab . At vere Fader og <b>Moder</b> er en guddommelig . Velsignelse , som Foreldrene stulle takke for ved en christelig Bsrneopdragelse ; — da vil Moderen , — paa hvem Bsrnenes Opdragelse nermest hviler — erfare <b>Sandheden</b> af 1 Tim . 2 , 15 : hun stal frelses ved Bsrnefsdsel . Forklar mig dette Sted . <b>Kvinden</b> lerer i sin Fsdselsmerte at raabe til Gud om Hjelp fra ham , — og ved Barnets Vugge lerer hun at bede , — og ved dets Vaare at tro og haabe . 2. Mand og <b>Kvinde</b> skulle vere ti ! hinandens gjensidige Hjelp i alle Livets Vilkaar . At de det ere , kunne I hver Dag i Eders eget Hjem se paa Eders Fader og <b>Moder</b> . De have Intet for sig alene , men Alt er dem felleds ; ' mangler den Ene Noget , saa erstatter den Anden det ; er den Ene syg , saa er den Anden til Hjelp . Og Enhver af dem har sine seregne Gaver . Manden stnl med sin Kraft beskytte <b>Konen</b> , og hun stal med sin <b>Kjerlighed</b> gjsre Mandens Liv lykkelig!. Hvad var altsaa Wgtestabets ' dobbelte Hensigt ? Valget . Da Forbindelsen mellem Mand og <b>Kvinde</b> i Wgtestabet varer for det hele Liv , saa er Valget meget vigtigt . „ Thi prove , hvo sig binder , om <b>Hjerte</b> Hjerte sinder ; thi Sandsens Rus

<h4>497</h4>hvad stulle I svare ? Af Tanker kommer der lettelig Tale og Handling . Og skulde det end ikke komme dertil , saa siger Christus Math . 5 , 28 : liver den , som ser paa en <b>Kvinde</b> for at begjere bende , har allerede bedrevet Hoer med hende i sit <b>Hjerte</b> . Heras se vi altsaa , at allerede Hjertets onde Lyst er Synd . Holder derfor Eders <b>Hjerte</b> rent kjere Bsrn ! Midler dertil ere 1. At Vaage over Hjertet . Merk vel , hvad der ere for Tanker derinde , om de ere rene eller urene . Lad det ikke vere dig ligcgyldigt , om du tenker paa , hvad der er Utugtigt , eller hvad der er Rent . 2. Naar ukydste Tanker komme uforvarende over dig , saa hengiv dig ikke til dem og udmal dem ikke for dig . Luther siger : Du kan ikke hindre Fuglen i at styve over dit Hoved , men du kan hindre ham iat bygge Rede i dine Håar . Hvad mente Luther dermed , nåar man anvender det paa de urene Tanker ? Serlig stal nu den Tnnke stadig vere hos dig , at den allestedsnerverende Gud stuer ind i dit <b>Hjerte</b> , og prsver Hjerter og Nyrer . Den hellige Frygt for den allestedsnerverende Gud . og <b>Kjerlighed</b> til ham stal afholde os fra at dvele ved utugtige Tanker , ' — men bringe os til strax at fordrive dem . 3. Undgna Anledninger , hvorved ukydste Tanker kunne fremkomme , f . Ex . slet Selstat » , eller hsre paa usedelig Tale , og lese daarlige Romaner , se letferdige Skuespil og Billeder o . s . v. <b>Oie</b> og Ore ere de Dsre , som saa lettelig kunne fsre dig bort fra Gnd og gjsre dit <b>Hjerte</b> urent . Hjertets Renhet » priser Jesus som en salig Ting , nåar han siger : Salige ere de rene af Hjertet , thi de stulle se ' Gud . At se Gud , som <b>selv</b> er ren og hellig , — bvilken stor Forjettetse ! 2. ' Kydst og tugtig i Ord . Math . 12 , 34 : Hvad Hjertet er fuldt af lsber Munoen over nf . Anvender nu det paa Ukydsthed i Hjertet . Usedelig og utugtig Tale er ikke alene vnnerende for det ene Menneske ligeoverfor det Andet , men Gud har i sit Ord udtryktelig forbudt det i Eph . 5 , 4 og 4 , 29 : ingen raaddcn Snak udgaa af Eders Muud , men saadan Tale , som er god til fornsden Opbyggclse . En saadan raaddcn Snak bedrsvcr Guds Aand , — og kunne I bringe det over Eders <b>Hjerte</b> , at bedrsue Guds Aand . som vil hjelpe Eder til Salighed ? Nei brug hellere din Mund tll at bede , love og takke . Vil du ikke af <b>Kjerlighed</b> til Gud lade dig afholde fra utugtig Tale , saa viid , hvad Herren siger Mth . 12 , 36 , at Menneskene stulle gjsre Regnstab paa Dommens Dag for hvert utilbsrligt Ord , som ' de have talet . Ved Siden af Ens egen Synd kommer nu dertil , nt man ogsaa forforer Andre ved sit stette Exempel . Kydst og tugtig i Gjerninger . Den . som ikke undser sig for Gud og Mennesker ved nt tale utugtige Ord , han vil heller ikke undse sig for at udsve ukydste og utugtige Gjerninger , og snnrt vil hau tabe stir Uskyldighet » . Vi ville nevne nogle Bibelsprog , som hsre herhen . 1 Kor . 6 , 15 : Vide I ikke , at ' Eders Legemer

<h4>603</h4>Samvittighet » ; han har i den hellige Daab givet os sin Hellig- Aand ; han har nedlagt en Lengsel efter ham i vort <b>Hjerte</b> ; og han vil fremdeles skjenke os sin Himmel . Men nu treder Synden hindrende imellem i dette Samfund ; den gjsr en Skilsmisse , som Skriften siger . Samfundet er forstyrret : Du hsrer ikke mere paa den guddommelige Stemme i din Samvittighet » ; du har ingen Lengsel efter ham , men har ganske andre Lyster ; du spsrger ikke efter hans Himmel , men efter Penge og snndselig og verdslig Glede . Gnd stjnler sit Ansigt for dig : Gud og hans hellige Villie forstunder mere og mere for dine <b>Oine</b> ; du bryter dig ikke om at hsre om ham , men vender dig bort fla dem , som sige til dig : du synder mod de ti Guds Bud . Vort Sprog forklarer ogsaa Synden fra den indre Side : Skilsmisse , Bortvend else fra Gud , det er Synd . Men hvor der er Bortvendelse , — der er der Henvendelse et andetsteds hen ; thi det menneskelige <b>Hjerte</b> og Sind er stabt for Samfund — og det kan og vil aldrig vere alene . Hvor vender da et Menneske sig hen , som vender sig bort fra Gud ? Til Verden og til det Forgjengelige . Hvorledes lerer Mennesket at forståa , at det har begnaetSnnd ? Af Loven . Ved Loven kommer Syndens Erkjendelse . Loven lerer os ret nt forståa , hvad Synd er . Loven viser nemlig , at Synden er Uret nermest imod Gud , som man for hans Mngts og Kjerligheds Skyld stal ere mest . Loven viser , at Synden er en Bortvendelse fra Gud , og i sin dybeste Grund et Had af Gud . Arvesynd . Er jeg nu en Synder , hvem Trudselen angaar eller er jeg En , som stlax kan gribe efter Forjettelsen ? Vi ville forst adspsrge Skriften . Ps . 51 , 7 : Se jeg er fsdt i Misgjerning , og min <b>Moder</b> har undfanget mig i Synd . Ps . 130 , 3 : Herre ! dersom du vil tnge vare paa Misgjerninger, Herre , hvo kan da beståa ? Ordspr . 20 , 9 : Hvo kan sige : jeg har renset mit <b>Hjerte</b> , jeg er ren nf min Synd ? Nom , 3 , 10. 12 : Der er ingen Retferdig , end ikke En ; alle ere afuegne . Rom . 3 , 23 : Alle have syndet , og dem fattes Guds 9 Ere : 1 Joh . 1 , 8 : Dersom vi sige : vi have ikke Synd . bedrage vi os <b>selv</b> . og <b>Sandheden</b> er ikke i os . Joh . 3. 6 : Hvad som er fsdt cif Kjsd er Kjsd . 1 Mosb . 8 , 21 : Menneskets <b>Hjertes</b> Tanke er ond af hans Ungdom . Alle disse Bibelsteder udsige , at alle Mennesker ere Syndere, og at ikke en Eneste er undtagen , ja vor Natur er lige fra Fsdselen af gjcnnemtrengt af Synd . Vi ere af Naturen Vredens Bsrn ( Eph . 2 , 3 ) . Den menneskelige Natur er saadan , at Gud har Mishag i den ; den er syndig , og den bliver

<h4>642</h4>og <b>Moder</b> , som helligbolder Hviledagen o . s . v. han er lydig . Men stal saadan Lydighet » kunne vente nogen Lsn us Gud , da maa den vere en Fslge afK jer lighed til Gud . Dette viser os tilbage til det fsrste Bud : jeg er din Gud , — og bvad laa deri ? Det fsler nu vel Mennesket , at Gud i <b>Sandhed</b> er dets Fader , og at det har alle Goder , de legemlige , aandelige og evige fra Gud alene . Af en saadan taknemmelig Fslelse fremvoxer <b>Kjerlighed</b> til Gud , sua Mennesket siger til ham : Nu vil jeg ogsaa gjerne vere dit Barn . vere dig lydig og holde dine Bud . ' Derfor siger Johannes: Den , som er fsdt af Gud . han synder ikke og kan ikke synde . Dette kan i Snndhed siges om den Herre Jesus , som sagde : Min Mad er , at jeg gjsr min Faders Villie . Saaledes stal ogsaa du og jeg sige . Og er det vort <b>Hjertes</b> Villie , saa vil alt , hvad vi gjsre , blive gode Gjerninger , baade vort Arbeide og vor Hvile , vore Bonner og vore Velgjerninger . Men uden <b>Kjerlighed</b> til Gud er Intet en god Gjerning og Intet kan vente nogen Lsn . Men et Beger koldt Vand af <b>Kjerlighed</b> til Christus stal ikke blive übelsnnet . " Men hvor findes de , som af ren <b>Kjerlighed</b> til Gud , kunne holde hans Bud ? Vi vide det vel : for Gud er Ingen retferdig : der er Ingen , som gjsr Godt , end ikke En ; de ere Alle afvegne . ' Af Naturen kan Mennesket ikke elste Gud , og altsaa heller ikke af <b>Kjerlighed</b> vere ham lydig . Men af sin Naade omdanner Gud Mennesket saaledes . at det kan vere ham lydigt . Epb . 2. 10 : Vi ere hans Verk , skaute i Christo Jesu til gode Gjerninger , til hvilke Gud forut » beredte os , at vi skulle vandre i dem . Omendstjsndt vi nltsya af Naturen ere Syndere , saa er det dog Guds Hensigt , ut vi skulle vaudre i gode Gjerninger . Men dertil maa Gud <b>selv</b> fsrst gssre os dygtige ; hau maa danue os til nye Mennesker . Og ved sin Ssn Jesus Christus danner han os til saadanne Mennesker , som af Kjellighed til Gud kunne gjsre gode Gjerninger . Gud har beredet Enhver af Eder til gode Gjerninger , kjere Bsrn , gjennem den hellige Dnab . Skriften kalder Daaben en god Samvittigheds Pagt med Gud . Ved Eders Daab og christelige Undervisning er Eders <b>Hjerte</b> omstå bt , saa at I kunne elste Gud og holde hans Bud . Joh 15 , 5 : hvo . som bliver i mig og jeg i ham , han berer megen Frugt . Efter en saadan Omstabelse af Hjertet ved Christum kan nu ogsaa Skriften forlange gode Gjerninger , — som den ogsaa forlanger paa hvert Blad . Col . 1 , 10 : Omgaaes verdigen for Herren til fuldt Velbehag og verer frugtbare paa al god Gjerning og voxer til Guds Erkjendelse . Gal . 6 , 9 : Men nåar vi gjsre det Gode , lader os ikke blive trette ; thi vi stulle og hsste i sin Tid , saafremt vi ikke forsage . Ise her , hvorledes Paulus setter de gode Gjerninger og Belsnningen sammen . Han var ogsaa <b>selv</b> den Mand , som af <b>Kjerlighed</b> til Gud og Christus redeligen strebte at holde Guds Bud . Du kan altsaa ikke undskylde

<h4>759</h4>David , Tobias , Elias , Daniel , Apostlene , Stefanus og Herren <b>selv</b> . Saadanne Lidelser , som Gud sender , kalde vi Christne et Kors , nåar de ere af stadig og lang Varighet ) , fordi saadanne Lidelser ligne Jesu Lidelser . Tsr vi nu ikke sige , at den , som har store Lidelser , er ugudelig , saa tsr vi ligesaa lidet sige , at deu , som undgnar dem ug nyder stor Lykke , er from . Pred . 8. 14 : Der ere Retferdige , hvilke det rammer efter de Ugudeliges Gjerning , og der ere Ugudelige , hvilke det rammer efter de Retferdiges Gjerning . Dette ville vi ikke kunne forståa , men maa overlade det til ham , som i sin urandsagelige Visdom udteler baade Lytte og Lidelse . Lidelsernes Storrelse . Lidelserne kunne ofte vere saa store , at det ser ud som om Gud ikke kunde vere Fader . Dette have alle Fromme erfarer . Derfor siger Hjob 30 , 20. 21 : leg striger til dig , men du svarer mig ikke ; du haver forvendt dig til at vere grum imod mig , du imodstaar mig af Had ved din Haands Styrke . Ps . 77 , 8. 9 : Skal Herren bortkaste i Evighed ; er hans Mistundhed ude <b>kvindelig</b> ? Ps . 69 , 4 : leg er bleven tret af det , jeg haver raabt , min Strube er hes ; mine <b>Oine</b> ere forteredc , idet jeg venter paa Gud . Jesus svcdede i Gethsemane Angstens Sved , saa hans Sved faldt som Blodsdraaber til Jorden , og paa Korset raabte han : min Gud , min Gud , hvi har du forladt mig ! Saaledes gaar det fremdeles . Herren tugter mest , hvem han har kjer . Og hvorfor gjsr han det ? 1 ) Din Tillid til Gud stal styrkes, som den kananeiste <b>Kvinde</b> , som ikke aflod at bede Frelseren , hvor haard han end anstillcdc sig . 2 ' ) Naar Nsden er rigtig stor , kan man bedst se den guddommelige Magts Herlighed . Naar vi fsrst intet Nnnd vide , saa hjelper Herren baade med Naad og Daad ; nnar Nsden er stsrst er Hjelpen nermest . 3 ) Gud kjender bedst den beleilige Time for sin Hjelp , og han vil sende den , nnar hans Visdom og hans <b>Kjerlighed</b> tilsiger ham det . Lid derfor , tro og haab . Gud stal gjsre en Ende pna din Lidelse . 4 ) Du forstaar ikke Guds Fsrelse med dig nu , men engang stal du lere at forståa den . Det lerte baade Hjob og David at forståa . Hust kun Herrens Ord Es . 49 , 15 : kan og en <b>Kvinde</b> glemme sit diende Barn , vil jeg dog ikke glemme dig ; og hans Forsikring i Es . 54. 7. 8 : leg hnr forladt dig et lidet Oieblik , men jeg vil samle dig med stor Barmhjertighet » ; det lidet Vreden vårede , skjulte jeg mit Ansigt et Oieblik for dig . men jeg haver forbarmet mig over dig med evig Mistundhed . Ps . 103 , 13 : Ligesom en Fader forbarmer sig over Bsrn , saa forbarmer Herren sig over dem , som frygte bam . „ Og varer det fra Morgenstund alt ind til Aftens Ende , Da stal mit <b>Hjerte</b> ingenlund mistrsstet bort sig vende . Sna gjsr Guds Folk af rette Art , Som han med Tugt har tvunget hart . De bie paa Guds Time . " En Christen maa have Lidelser . Den Christne stal herlig-

<h4>1132</h4>have vi Tolderen : Gud ver mig arme Synder naadig . Og hans Bsn blev hsrt : han gik retferdiggjort hjem til sit Hus . Bodsbsnnen er den svereste Bsn . Thi Syndernes Erkjendelse og Anger er en saa sver Ting . En Syndsbckjendelsc vil saa nsdig gaa over Leverne ; bestandig ved det onde <b>Hjerte</b> Noget at finde til sin Undflyldning , eller det kaster Skylden over paa Andre . Men det gjorde ilke de fire Mend , vi have omtalt . En Bodsbsn staar ned og reiser op. Esra falder paa sine Kne , David falder ned for Guds Aasyn , Tolderen staar sig for Brystet og tsr ikke oplsfte sine <b>Oine</b> , thi de ere sig Alle bevidste ved sine Synder at have bedrsvet den guddommelige <b>Kjerlighed</b> . I denne Fslelse vilde Synderen fortvivle , dersom han ikke ogsaa havde en Fslelse af den guddommelige Naade og Trsst , som reiser Synderen op. Er der hos os megen Synd , saa er der hos Gud end mere Naade . Han vil ikke forstode den , der kommer til ham med et bodferdigt <b>Hjerte</b> , og som siger til ham : Herre , jeg har handlet ilde ! Syndens Magt blev mig for stor . Veien jeg ei vandre vilde . Den du viser i dit Ord ; Og hvor findes nu den Hule , Hvor min Skam jeg kunde skjule ? ( Landstad No . 399 ) . 4. Lov og Takkebsn . Ps . 8 , 2 : Herre , vort Herstab ! hvor herligt er dit Naon over al Jorden ! Dette er en Tale til Gud , altsaa en Bsn , men ingen Bedebsn , ingen Bodsbsn , ingen Tak . Det er en Lov d . e . en Prisen af Guds Storhed og Herlighed . En saadan Lovprisning have vi i Skriften , Psalme 104. Kirken maa ofte ligesom og den Enkelte gjsre Brug af Lovsang , og i den bekjendte Psalme „ O store Gud vi love dig " huve vi Exempel paa en saadan . Hvad en Takkebsn er udsiger Navnet <b>selv</b> . Al Tak til Gud er et Udtryk for den Erkjendelse , at vi have denne eller hin Gave fra Gud . I Psalmerne hedder det derfor ofte : jeg takker dig Herre , min Gud . At takke Gud stal vere vor Lyst og Glede som det har veret det for alle de hellige Mend . Ps . 92 , 2. 3 : Det er godt at takke Herren , og at synge dit Navn Psalmer du Hsieste ! at kundgjsre din Mistundhed om Morgenen og din <b>Sandhed</b> om Netterne . I Skriften er os opbevaret herlige Takkebsnner af Debora , Harma og Maria , — og kunne I sige mig ved hvilke Anledninger . Nuar vi paa engang love og takke Gud , saa siges vi at prise ham . Jeg priser dig Herre er det samme som : jeg crkjender og lover din Storhed og Majestet , og erkjender at zeg har Alt ved dig . Vi have mange herlige Lov og Takkesange . Den , som vi almindelig gjsr Brug af ved Takkefester : „ Nu takker Alle Gud " er affattet efter Sir . 50 , 29 - 31.

<h4>1161</h4>Vi have just seet , at Gud ikke hsrer « bodferdige Synderes Bsnner . Derimot » forlanger Skriften , nt en Bsn stal vere andegtig, ydmyg , tloende og fnmodig . Dette ville vi nsiere omtale . 1. Andoegtig . At vere audegtig vil sige , at have sit Sind oplsftet til Gud og tenke paa . at du taler med Gud . En from Maud har sagt ! Gud hsrer ikke Bsnneus Ord , nnar ikke den Bedende <b>selv</b> har hsrt dem . Saaledes andegtig var Harma , Samuels <b>Moder</b> ; hun var med alle sine Tanker ved sin Bsn ; en andegtig Bsn er en alvorlig Bsn , og om den siger Jacob 5 , 16 : en Retferdigs Bsn formaar meget , ' naar den er alvorlig . Det modsatte af eu alvorlig Bsn er en letsindig Bsu , hvor Ordene kun udtales med Munden , men hvor Tankerne og Hjertet ere langt borte . En saadan Bsn er desverre kun altfor almindelig . Jesus forbyder eu saadan Bsn i Mth . 6 , 7 , idet han siger : Naar I bede , stulle I ikke bruge overftsdige Ord , som Hedningelne ; thi de mene at blive bsnhsrte , nåar de bruge mange Ord . En Bsn med mange Ord er ofte en tankelss Bsn , og hvorledes skulde du forlange , at Gud skulde vere tilstede ved den Bsn , hvor du <b>selv</b> ikke er ? Gud er eu Aaud , siger Christus . og de , som ham tilbede , bsr det at tilbede i Aand og <b>Sandhed</b> . Han vil altsaa have Oprigtighed i Bsnucn . saa at Hjette og Old stemme inderlig overens . Hvor det er Tilfeltet , der er Bsnnen hjertelig . En saadan Bsn forlanger Jer . Vegr . 3 , 4 l : Lader os oplsfte dort <b>Hjerte</b> med Hender til Gud i Himmelen . Denmod siger Mth . 15 , 8 : Dette Folk holder sig ner til mig med sin Mund og erer mig med Leberne , men deres <b>Hjerte</b> er langt fra mig . Og saadanne kalder Jesus Hyklere . Kunue I altsaa uu sige mig , hvorledes man kan vere en Hykler , nanr man beder ? Og nanr er en Bsn andegtig ? Giv da din Naade til Herre Jesus , at vi ikke aleue maa bede med Munden , men af vort hele og fulde <b>Hjerte</b> . 2. Idmyg . Ps . 34. 19 : Herren er ner hos dem , som have et ssnderbrudt <b>Hjerte</b> , og vil frelse dem , som have en ssnderbrudt Aaud . Et saadaut <b>Hjerte</b> og et saadant Sind kan alene komme af den Bevidsthed , at vi ere Syndere for Gud . som have fortjent Straf af ham ug som Intet kunne fordre , men som Betlere komme for Gud med deres Bsn . En saadan ydmyg Beder har Abraham veret 1 Moscb . 18 , 27 : Se jeg har begyndt at tale , alligevel jeg er Stsv og Aste . Jacob : Herre , jeg er for ringe til al div Mistundhed . Daniel . Tolderen , som stod langt borte og ikke engang vilde loste sine <b>Oine</b> op imod Himmelen , men kuv stog sig

<h4>1439</h4>hvor rig er jeg da bleven , hvor er jeg benaadiget ! Mine Synder ere mig forladte . Deu Herre Jesus har elsket mig arme Menneske mere end Fader og <b>Moder</b> . Saa vil jeg ogsaa takke ham mere end jeg takker Fader og <b>Moder</b> ; blive end Alle utro mod dig , jeg vil dog vere dig tro ; jeg vil vise dig , at Taknemmelighed dog ikke er uddsd paa Jorden ; du led for mig , du gik i Dsden for mig , saa vil jeg evig tjene dig . Saa stal det opfyltes paa mig . hvad Frelseren siger Joh . 6 , 56 : Hvo , som eder mit Kjsd og drikker mit Blod , han bliver i mig og jeg i ham . Vi se Heras , " at den hellige Nadver er ikke blot en Erindring om Jesu Lidelse , som de fromme og taknemmelige Christne ogsaa ellers daglig ville ihukomme ; men han har personlig forbundet fig med mig ; hnn har i Nadveren i <b>Sandhed</b> veret tilstede , jeg har under Brsd og Vin modtaget hans Legeme og Blod , han har <b>selv</b> bespist mig med sig <b>selv</b> . Nu strsmmer ogsaa Alt , som hsrer til gudfrygtigt Liv og gudftygtig Vandel over fra ham , som er Vinstokken og ind i mig , som er Grenen . Imod Synden formaar du nu Alt ved ham , som i sit Maaltid har styrket dig : Jesus Christus . Saa vil du ogsaa i dit Liv som Christus i sit vise Hengivenhed i Guds Villie , Taalmodighed i Lidelse og Tillid til Gud . ' Af Taknemmelighed vil du nu vere din Herre Jesu lydig . Se dette er Frugten af din Tro efter den hellige Nadver . Og kommer denne Frugt , saa har du veret en verdig Gjest ved Nadveren ; kommer den ikke , saa er det meget at befrygte , at du ikke har veret det . Thi 1 Joh . 2 , 6 siger : Hvo , som siger , at han bliver i mig , hau er og skyldig til at vandre saaledes . som han vandrede . Og den samme Apostel siger : den , som bliver i ham , synder ikke ; den , som synder , har ikke seet og ikke erkjendt hnm . Naar de Christne i Kirkens fsrste Tider nsde Nadveren pleiede man at sige : det Hellige er for de Hellige . Og man mente dermed , at den hellige Nadver er for dem , som ville helliggjore sig . Og denne Kraft til at helliggjsre vil Nadveren uetop give . Taknemmeligheden mod lesum lader dem i enhver Henseende tåge det nsie , saa de ere samvitiighedsfulde i sin Tjeneste og ikke bedrsve den kjere Herre lesum . <b>Kjerlighed</b> til Nesten . Troen paa : at mine Synder . ere forladte bevirker ved Siden af Taknemmelighed mod lesnm ogsaa Lydighcd unod hans Bud . Det taknemmelige <b>Hjerte</b> spsrger : Herre , bvad vil du at jeg stal gjsre ? Da viser Herren os bort fra sig <b>selv</b> og heu til vor Neste og siger : hvadsomhelst I have gjort mod en af disse mine mindste Brsdre have I gjort mod mig . Alt , som vi skulle gjsre imod Nesten , er sammenfattet i Budet om <b>Kjerlighed</b> . 1 Kor . 10. 16 siger , at vi ved den velsignede Kalk og det velsignede Brsd trede i Samfund med Christo . Det hedder derpaa i Vers 17 : et Brsd , et Legeme ere vi Mange ; thi vi ere Alle blevne delagtige i det ene Brsd . „ De Mange " en her vore Medchristne , de udgfsre et Legeme , d . e . et Samfund , — et Samfund , som netop

<h4>1509</h4>— idag stal intet Ord hsres derom , idag staar Lereren fuldkommen hevet derover ved sin <b>Kjerlighed</b> til Bsrnenes udsdeligc Sjel . Saadan faderlig <b>Kjerlighed</b> sinder de rette Ord . og dette Ord vil ogsaa ved Guds Naade finde et godt Sted i Bsrnenes <b>Hjerte</b> . Den Fslelse , som i dette Oieblik stal besjele ham , Bsrnenes Fader , har Johannes udtalt i 3 Joh . 4 : Jeg har ingen stsrre Glede end denne , at jeg hsrer mine Vsrn vandre i <b>Sandhed</b> . — Lereren el fremdeles Herrens Tjener , og hver Morgen har han paany hort det Raab : Vogt mine Lam . Som deres Lerer , dertil indsat af Herren <b>selv</b> , fsrer han nu deres <b>Blik</b> tilbage og lader dem se al den <b>Kjerlighed</b> , Naade og Trofasthet » , som Herren i de mange Aar i Skolen har ladet dem vere en Gjenstand for . Han viser dem derved bort fra sig . <b>selv</b> og Foreldrene og hen til Guds <b>Kjerlighed</b> i Christo . den sterkeste Magt til at holde dem fast . — Bsrnene give ogsaa Lereren dens Haand , men kun til Afsted , og til Tak til Lereren for hvad , han har veret for dem ; men Lereren vogte sig vel for med dette Haandstag at ville forlange noget Lsfte ; det tilkommer ikke ham . — Til Afsted er det smukt , at Lereren giver enhver af Bsrnene et Bibelsprog . afpasset efter ethvert Barns ' Forhold og Gemytsbestaffenhed . Det er det bedste , han kan give dem , og senere i Livet vil et saadant Bibelsprog kunne blive en rig Skat for Barnet . At Foreldrene ere tilstede ved Confirmantafsteden i Skolen er ikke snsteligt . Det har sine Farer baade for Lereren og for Bsrnene, og denne Skolefest taber da den private Karakter , som den bsr ssge at bibeholde . Saa meget snsteligere er det derimot » , at de Elever , som endnu stulle blive tilbage i Skolen , ere tilstede . Man maa altid erindre , at Afsteden med Confirmanterne hsrer Skolen og ikke Kirken til . En ganste anden Sag er det med selve Consirmationen. Der hsre Foreldrene hjemme , baade paa Grund af deres naturlige Forhold til Bsrnene og paa Grund af deres Stilling som Menighedslemmer . Som saadanne skulle de nu hsre til , og <b>selv</b> overbevise sig om , at Vorrenes Kundstab i Chliftendommen er saa stor , at de kunne optages iblandt Menighedens myndige Lemmer . Men i Skolen bsr Lerer og Elever vere stille for sig og for Gud , sna bliver det unge <b>Hjerte</b> ikke forstyrret ved noget Mre , men er villigt til alene at aabne sig for den kjen Lerers sidste Ord . Ogsaa Lereren vil vide at iudrette sin Tiltale til Bsrnene hjerteligere og simplere , nåar de Voxne ikke ere tilstede . Lad derimud Skolens tilbagevennde Elever , serlig de eldre af dem , vere tilstede . For dem vil det vere gavnligt og nyttigt ; de ville derved lere det Maal ak kjende , som ogsaa er sat for deres Skolegang . — At Bsntten ved denne Skolefest maa indtage en gcniste vesentlig Plads , bchsver vel neppe at siges den troende Lerer .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016031548116'target=_>Landstad, M.B., 1871, Kirkesalmebog</a><h4>2151</h4>Til Hoimesse . Som : Kom hid til mig enhver iscer . 495. Paa Jorden er en liden Hjord , Som haver for sig Herrens Ord Udi den kristne Kirke , Men deriblandt man finde stal Til denne Dag et ringe Tal , Som Gud i <b>Sandhed</b> dyrke . 2. Hor , Menneske ! dit kristen Navn Kun da er dig til Trost og Gavn , Naar du som Kristen lever ; Hvi pryder du dermed din Sag , Og gjor dog ei paa Naadens Dag , Hvad Gud af dig udkrcever ? 3. Du roser dig af Daabens Pagt , Men ilde tager du i Agt , At du i Naaden bliver ; Endnu du ikke fare lod Synd , Satan , Verden , Kjod og Blod , Det Onde , du bedriver . 4. Du horer vel Guds sande Ord , Du kommer og til Herrens Bord , Og tager Kristi Gaver , Men hvor er dog den sode Lugt , Som er den Bedrings gode Frugt , De sande Kristne haver ? 5. Din Bon gaar spildt udaf din Mund , Naar den ei gaar af Hjertens Grund Og ud af Troens Kilde ; Dit Herre ! Herre ! nytter ei , Naar du vil gaa din egen Vei , Som er at gjore ilde . 6. Hvor herlig er du bryllupsklart ? Hvor er din Tro , Retfærdighed , Og et taatmodigt <b>Hjerte</b> ? Hvor er dog din sagtmodig Aand , Din <b>Kjærlighed</b> og Hjælpehaand Udi din Ncestes Smerte ? 7. Er du end skjult for Mennesker , Dog Herrens Vine grant dig ser , I Vrede han dig <b>moder</b> ; Vil du ei lade Hykleri Og dig fra Synden gjore fri , Til Helved han dig stoder . 8. O , derfor fog den Herre Krist , Og vogt dig vel for Djcrvlens List , Til Verden dig ei hænge ; Forncegt dig <b>selv</b> , korsfest dit Kjod , Med Von og Haab i Nsd og Dod Sog ind til Gud at troenge .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014121208040'target=_>Ahlfeld, Friedrich, 1878, Menneskelivet i Guds Ords Lys</a><h4>303</h4>saaledes vil det ogsaa gaae med dine Born . I den tidlige Barndom er Hjertet blsdt og Hukommelsen ung . Hvad man da lcrrer , det indprceger sig dybt og glemmes fordetmeste aldrig . Men ikke nok hermed . Hvad vi lcere i vore forste Barneaar . og navnlig af vor <b>Moder</b> , det seer anderledes ud end hvad vi senere lcere . Morgenduggen hamger ved det : Moderkærligheden med den hele Erindring om disse lykkelige , sorgfrie Nar omsvcever det . Naar vi senere trcrffe saadanns Udsagn og Fortællinger igjen , tale de et ganste andet Sprog end de , vi siden have lcert . Dette ligger ikke i Guds Ord , som om nogen Deel af samme vare kraftigere end en anden . Det ligger i Tiden og Hjertets Modtagelighed. Disse forste Udsagn og Fortællinger have staaet Nodder i hele Livet . Saaledes er det ogsaa med Sceden ude paa Ageren . Hvad der saaes tidligt om Vaaren , fcester dybest Rod . Om ogsaa en anden Ager senere besaaes med samme Sced , modstaaer hiin dog bedre Tyrken og lonner Landmanden med en rigers Nfgrode . Hvad der fra fsrst af er indprceget i Barnets <b>Hjerte</b> , giver dets Liv dets Retning . Historien kjender en Mand , fom allereds i sine tidligste Aar legede med en Kanon . Siden arbeidede han ncesten hele sit Liv igjennem med dette Mordredskab . Da er det bedre , at vi tidligt mods Herren ; da kunne vi ikke vride os fra Hans Dragelse , saalcenge vi leve . Ja , om end vi seners fare vild en Stund , om end Livets <b>Sand</b> og Stsv Icegger sig paa hins Scedekorn , faa er der dog fremdeles Liv i dem , og i sin Tid komme dog de forst op. Intet har oftere fort forvildede og i Elendighed nedfunkne Syndere paa Omvendelsens Vei end Tilbageblikket paa en gudfrygtig og lykkelig Ungdom og Ihukommelfen af de Stykker af Guds Ord , som de havde lcert i sin Barndom . Ja , netop de ere for Mange blevne en Kjcep og Stav i Dsdsstyggens Dal . Men hellerikke for Eder Forceldre bliver et saadant Arbeide paa Eders Borns Frelfe uden Frugt . Kun den Fader og <b>Moder</b> besidder i Bsrnenes Vine den rette Vcerdighed , fom har den himmelske Fader staaende bagved sig , som forretter sit Kald i Hans Navn og henvifer sine Bsrn til Ham . Da hviler der en Glands af guddommelig Majestcet om deres Hoveder , som fremkalder en langt stsrre Wrefrygt og en langt sandere Lydighed end al blodagtig <b>Kjcerlighed</b> og alle strengs Befalinger . Dette er imidlertid en Frugt , som falder mere udad , medens derimod en anden falder i dit eget <b>Hjerte</b> Jo eenfoldigere vi tage Guds Ord , desto stsrre Magt har det . Hvo fom <b>selv</b> er ret liden og eenfoldig , idet han meddeler sine Bsrn det , hvo som nedlader sig til dem , ligesom Gud Herren i Ordet <b>selv</b> har nedladt sig til os , han modtager

<h4>361</h4>hvilken Troskab Han arbeider paa Peter ! Hvor ofte taler Han ikke <b>kjcerligt</b> , hvor ofte straffende til ham ! I Hans Venlighed er der <b>Kjcerlighed</b> , og i Hans Straf er der ogsaa KjcrrKghed . Da Peter havde fornegtet Ham tre Gange , og han i sin fcengslede Tilstand ikke lcrnger med Ord kunde ndtale sin <b>Kjcerlighed</b> , ndtalte Han den med etOiekast . Han rammede den faldne Discipel i Hjertet , og strax lob Draaber fra dette ned over hans Kinder . Efter sin Opstandelse ssgte Han ingen af Disciplene med saa vrcrndende Iver som den iamme Peter . O , I Lcrrere , lcerer her af Eders Mester , den fra Gud komne Lcerer , hvorledes man stal bcere de Skrobelige , hvorledes man stal gaae efter Sjelene med <b>Kjcrrlighed</b> til sin Tid og med Alvor til sin Tid ! O , stoder intet Barn bort ! Dommer ikke om et eneste af dem , at det er redmngslost fortabt ! Eders Kald er jo et Fader- og Moderkald . Gjorer fom en <b>Moder</b> , der ofrer den meste <b>Kjcerlighed</b> paa det svageste Barn ! Drager Bornene med Venlighed og Straf , saaledes som Eders Herre gjorde ! Maatte Han ogsaa give Eder et <b>Oie</b> , fnldt af <b>Kjcerlighed</b> og Sorg , der farer ind i Hjertet som en Krog , for ikke mere at flippe det ! Og naar et Barn formedelst sine ringe Evner kun gjor smaae Fremskridt i de forskjellige Underviisningsfag , saa seer til , at I ialfald lcegge en god Grundvold hos det i Hovedsagen , i Frelsens Kundskab ! Forsommer ikke at bede med Boruene og for Bornene ! Naar Landmanden har saaet , oplofter han sine Hcender og beder om Regn . Herren maa vcede det Ploiede og vande Scrden . Ogfaa I faae , saae hver Dag , Sommer og Vinter . Gaaer ikke bort uden at oplofte Eders <b>Oine</b> til de hellige Skyer ! Herren maa vande . Der kan jo Ingen kalde lesum Herre uden ved den Hellig-Aand . Al Tro og al Helliggjsrelse er en Naadesgjerning, og derfor maa man bede om den . Beder ogfaa Herren flittigt om at Han ved den Hellig-Aand vil paatrykke Eder sit Billede ! Ncest Forceldre , Faddere og Slagtninger ere I Barnets ncermeste Paarorende . I ere de Forste , som komme det imsde , naar det fsrstegang vandrer ud af Fcrdrehjemmet . Agtelsen for Lcereren er en Ledestjerne for Barnet . Han stal i sin Fcerd og alt sit Vcesen tjene Barnet til et Forbillede . O , maatte dog alle Born i sin Lcerer mode en <b>sand</b> Jesu Bekjender og Efterfolger ! Maatte et saadant Billede afprcege sig i ethvert Barnehjertes blode Voxtavle ! Hvilken Velsignelse vilde ikke derved komme over Ungdommen ! Og omvendt , hvilken Skade kan ikke en vantro Lcerer afstedkomme ved Tale og Levnet ! Han er istand til at berove Bornene deres Ungdoms helligste Glcede og at ndsaae en ond Sced i deres <b>Hjerte</b> , som

<h4>376</h4>Herre Jesus Christus ! vi takke Dig , fordi Du af ufortjent Nllllde og Miskundhed er bleven vor Fraende og Hjertensven. Hos Dig er <b>Kjcerligheden</b> og Venskabet uforfalsket og ufminket . Vi have ikke stjcenket Dig Noget , men Du os . Du har stjcenket os Alt ; vi derimod kunne ikke skjcrnke Dig Andet end et fattigt , skyldbetynget <b>Hjerte</b> . Du tager imod det og giver os din Retfærdighed igjen . Ja , Herre , faaledes har Dn stiftet Klceder med os , som Jonathan med David . Da Du bar dit Kors , svedende og segnende under Byrden og bedcekket med Skjcendsel og Foragt , da Du hang paa Korset , suld as Blod og Saar , Smerte og Torst , da bar Du vort Klcedebon . Og naar vi i Lonkammeret eller fom Skrifteborn og Nadoerdsgjcrster anamme Syndernes Forladelse , ja , naar vi engang fremtrcode retfærdige og hellige for din himmelske Faders Aasyn , da bcere vi dit Klcedebon . O , hav Tak for dette dit Vennehjerte ! O , bevar os i dit Samfund , faa at Du aldrig stal komme til at tiltale os for fidste Gang som Judas med dit „ Ven ! " Vi oille vcere dine Venner ial Evighed . Men giv os tillige , kjcrre Herre , i din Kirke her paa Jorden trofaste Venner , som redeligen understotte os i vor Saligheds Sag . Herre , Bornene , giv de Gamle saadanne Venner . Lad Ingen ' taa eensom . Forbarm Dig over os , saa at <b>selv</b> den Eensomste, som ingen Familie har , dog maa have Een , som bcerer og beder med ham , som staar ved hans Sygeleie , som i hans sidste Stund trykker hans <b>Oine</b> til , som udgyder en Taare ved hans Ligkiste og beder et Fadervor ved hans Grav . Du sande Sjeleven , ovvcek Du os af  saadanne Venner , og helliggjor os , saa vi beholde dem indtil Enden ! Amen . At ftaae eemom her i Verden er en tnng Skjebne ! et Menneske har vanstcligt for at bcere den . Naar man seer et eenligt Tyr paa Grcesgcmgen , en eenlig Blomst i Haven om Hosten eller et eenligt Trce , stallende paa en Bakkeø betages man af en vemodig Fornemmelse . Trcrerne forstaae vel ikke hverandres Susen og Hvisken : men det ene bestjcrrmer og bevarer dog det andet i " Stormen . Hvor meget storre er ikke Tranaen til et hjerteligt og broderligt Samfund blandt de Christne ! hvor meget mere trykkende er ikke Een>omdeden for dem ! Vistnok er en <b>sand</b> Christen aldrig ene eller forladt : han synger :

<h4>745</h4>Herre Jesus Christus , lad aldrig Brud og Brudgom , lad aldrig Mand og <b>Kvinde</b> glemme , at Du er deres forste Brudgom . Du har elstst os med en evig <b>Kjcerlighed</b> oq draget os med Miskundhed . Du har beilet til os med en Kjcerllghed , hvis Lige aldrig er opkommet i noget Menneska <b>Hjerte</b> . I den hellige Daab satte du Trolovelsesringen paa vor finger Med din tilgivende , fsgende , staanende og bcrrende Kærlighed vil Du drage os stedse ncermere til Diq O Herre gw os Naade til at lade os drage , til med hver Nag at dttve stedse ncermere forenede med Dig . Giv ethvert trolovet Par , at de Begge maae trolove sig stedse inderligere med Dlg , blive uadstillelige fra Dig og saaledes ogsaa vcere indbyrdes forbundne i een og samme Tro , een og samme Frelser , een og samme Retfærdighed og Helliggjorelfe oq esn og mmme Vandrmg til det Jerusalem heroventil . Ak Zerre lad dm Forbarmelse og Saligheden i Dig kaste sit <b>Blik</b> , sit Morgensur i enhver Forlovelsesstand , for at det ogsaa kan og opklare Huset tilligemed dets Nu as Naads vore forlovede Par ! Amen vor Text sammenllgner Gud sin Glcede over sit troe udvalgte med en Brudgoms Glceds over sin Brud . Vor Herre og frelser kalder sig <b>selv</b> Brudgommen og Menigheden Bruden den Dsber er Brudgommens Talsmand og Bruder Del ? ' fgr ? / svakte og i Jordan dMe Siele forer han t : l Herren , Brudgommen . Herren sammen bgner de Dage , Han tilbringer her paa Jorden , med den Nd Brudgommen er hos Bryllupsgæsterne . Bryllupsgicestirne saalcenge Brndgommen er hos dem Ja hele den Dd , hvori Kirken er stridende , hvori den forvenler Herrens Genkomst for at fuldende sit Frelsers k , amme

<h4>835</h4>ncermere end en Vens med sin Ven , en Broders med sin Broder , et Barns med sine Forældre ! Ak , Herre , Herre ! Algtefolk skulle vcere Eet , stulle vcere eet Kjod . O , gjor Du dem ret enige ! Bed Du for ethvert christeligt Mgtepar . Lad den Hellig-Aand arbeids paa dem for at drcebe deres Selvraadighed og Egensindighed , deres Hjerters Knlde og Hovmod . Drag dem Begge ind i dit Ord og din Nand . Lad det Ord fra Alteret : „ Hvad Gud har sammenfoiet , stal intet Menneske adstille " , stedse mere og mere blive <b>Sandhed</b> hos dem . Herre ! hellerikke den gamle Adam med sin Egen- Mie og Trodsighed bor adstille dem . Hellerikke i Hjerterne bor det komme til nogen Adskillelse og Kulde . Giv tvertimod Naads til at ethvert Par maa blive stedse fastere sammenfoiet i Dig , maa faae Dig og hinanden stedse kjcerere og hverken kunne stilles fra Dig eller hinanden . Giv dem dertil Alvor , Aarvaagenhed , Bon , Forsigtighed , Idmvghed og Taalmodighed. Lad dit Ord boe i ethvert Huus og Dag for Dag blive kraftigere i Hjerterne . Formedelst din hellige Fsdsel , formedelst din Trolovelse med vor Slcegt ville Du stjcenke os denne Naaoe ! Amen . Ved Alteret er Wgteparret viet og saaledes efter Guds Orden blevet Eet . Begge Parter have <b>selv</b> valgt hinanden og erklceret , at de elske hinanden , ja elske hinanden hoiere end noget andet Menneste . De have givet hinanden det hellige Lofte at ville vandre Pilegrimsveien gjennem dette jordiske Liv i Fællesskab og dels Sorg og Gloede med hinanden. De boe i Huus sammen ; de staae i et inderligere og fortroligere Forhold til hinanden indbyrdes end til andre Mennesker . Og dog er der mangfoldige Par , hos hvilke der aldrig kommer til nogen fand Enighed . Vi ville nu slet ikke tale om de Wgtefolk , fom kun en kold Beregning har fort fammen , i hvilket Tilfcelde det faaledes egentlig ikke er Personerne , men deres Midler , som ere blevne sammenviede. Men <b>selv</b> mellem dem , som have rakt hinanden Haanden af eget Valg og naturlig Kærlighed , hersker der ofte ingen Esndrcegtighed . Den <b>Kjoerlighed</b> mellem Forloveds, som svceveds i Skyerne , den drommende og fvcermensse <b>Kjcerlighed</b> , der heller vilde doe end give Slip paa den Elsteds , hvor fnart kjslner ikke ofte den ! Mikal , Sauls anden Datter , havde elsket David . Den unge Helt var en Mand efter hendes <b>Hjerte</b> . Zun cendssde ikke M Fadere Vrede ; hun blev Davids <b>Hustru</b> . Da Saul vilde drcebe ham , frelste hun hans Liv . Da Morderne trcengts md i Huset , havde hun allerede hjulpet David til at flygte . Ved at lede i Sengen fandt Fienderne et Afgudsbillede , lom Mikal havde lagt der istedetsor ham . Da David omstder var

<h4>863</h4>og Hjoelv . Hvor To ville herske , er der bestandig Ufred ; et godt Raad , en venlig Hjcelp og Tjeneste er derimod stedse velkommen . Derhos bor enhver Svigermoder vel betcenke , at Herren faa at sige nu giver hende Afsked og scetter hende ud af Virksomhed , for at hun , endnu forinden hendes sidste Time kommer , kan indgaae tll Guds Hvile . Salig er den , hvem Gud efter Dagens Sved og Moie stjcenker en rolig og stjon Aften , hvori han kan overstue sin Fortid , soge Naade for sine Synder , prife Gud og forberede sig til den Nat , da Ingen kan arbeide . Fra Mandens <b>Moder</b> komme vi til Konens Forceldre . Med Formaning til hjertelig Idmyghed bor de udsende sin Datter til hendes nye Huus . De bor ikke forlange , at Alting der stal vcrre ordnet og indrettet som i deres eget Huus . Det er Mandens Huus . Den unge <b>Kone</b> bor anerkjende og agte den Skik , som dersteds hersker , forsaavidt som den ikke er syndig . Hun er en Kvist , som er afstaaren af deres Huus ' s Stamme og indpodet ien anden . Podekvisten kan ikke medbringe den gamle Stammes Rodder ; den bliver sammenforet med en ny og fat i Forbindelse med dens Rodder . Og fremfor Alt bor den unge Kones Forceldre vogte sig for at ville holde hendes <b>Hjerte</b> tilbage i sit eget Huus og scette hende i en los Stilling , hvori hun ligesom " svcever midt imellem degge Familier . Hun har jo hengivet sig ganske til sin Mand , og hendes Forceldre have <b>selv</b> bortgivet hende til ham . Der gives intet Huus , som ikke har sine Sorger og Ojenvordigheder . Ogsaa til dem er hun viet . Forceldrenes Huus bor ikke ucere et Sted , hvor hun kan gaae hen og uoose sine Klager , og hvorfra hun vender tilbage til sit eget Huus som fra sit rette Hjem . Se til de Gudfrygtige i den gamle Pagt , ja endog for den gamle Pagt hvor straaler de ikke ved Siden af mange Christne i vor Tid ; Bethuel og hans Born betragte Rebekka som fuldstcenoig skilt fra deres Stamme ved sit Giftermaal med Isak . Ved Afskeden sige de ikke et Ord om at hun skal kunne soge nogen Tilflugt i sine Forceldres Huus . Hun maa tage sit hele <b>Hjerte</b> med sig til det fremmede Land og sit nye Hjem . Den unge Tobias ' s Svigerforældre , Raguel og hans <b>Hustru</b> Edua , formane sin Datter ved Afskeden til at cere sin Mands Forceldre som sine egne . Ikkedestomindre kan og stal den unge <b>Kone</b> fremdeles bevare sin <b>Kjcerlighed</b> og Wrefrygt for nne Forceldre . Ja , dette vil i faa meget hoiere Grad vcere Tilfceldet , naar de ved sin Viisdom og Erfaring bidrage til at berede hende Sted i hendes nye Hjem . Hun er nu forst og fremst sin Mands <b>Hustru</b> og faa derncrst deres Barn . For det Tredie bor Manden med <b>sand</b> Klogstab

<h4>1056</h4>De Fattige derimod gaae indad den og lads sig gjore rige af Herren . Istedetfor at knurre over din Fattigdom wr du heller takke Herren for at Han har bygget din Hytte saa ncer ved Frelsens Port , og at hans Indbydelser lyder saa klart og tydeligt derind til dig . Hold du kun fast ved din Frelser . Vederkvceg du dig og Dine med Guds gode Ord . O , hvor deilige blive ikke Aftenerne i den fattige Hytte , naar Faderen sidder og lceser hoit i <b>Sandhedens</b> evige Ord , og <b>Kone</b> og Born begjærlige og glade lytte derpaa ! Da opfyldes den lille Menighed med den Hellig-Aand ; Hjerterne udvide sig : den dunkle Lampe brcender klarere ; Fattigdommen glemmes ; I fryde Eder i Herren og Hans Styrke ; Guds <b>Kjcerlighed</b> er udost over Eder ; I fole og ane , at I have en usigeligt rig Arv . Lad derhos Intetsomhelst stjcele de velsignede Sondage fra dig . Du veed , at de fremfor alle andre Dage ere rige paa Herrens Naade . Paa dem skinner det evige Livs Morgenrode ind i din Hytte ; de ere dig et Pant paa den store Sabbat , paa den store Hvile hos Gud . Ssndagen udsoner dig med alle andre Stcender . Alle Stcender i Menigheden hore det samme Ords Forkyndelse , have den samme Frelser , ydmyge sig for Gud i den famme Syndsbekjendelse , olme retfærdiggjorte ved den famme Christi Fortjeneste , ere deelagtlge i det samme Christi Legeme og Blod og have det samme evige Livs Haab . Da gaaer den Opstandnes Rost gennem Menigheden og raaber : „ Fred vcere med Eder ! I m : g er der hverken Mand eller <b>Kvinde</b> , hverken lode eller Grceker , hverken Trcel eller Fri , hverken Fattig eller Mig . De , som ere i Christo , ere nye Skabninger " . Med Hensyn til dit Huusvcesen , da Voer tilfreds med hvad Herreu fljcenker dig . Ingens Liv bestaaer i hans Gods . i det , at han har Overflod . Eisr du Ydmyghed og Beskedenhed , saa er du ligesaa ng som den Rige . Den Rige kan hellerikke mere end spise sig most ; han kan hellerikke boe i mere end eet Huus ; han bcerer hellerikke mere end een Klcedning ad Gan- hellerikke i mere end een Seng ; han faaer m , id » t hellerikke mere end een Ligkiste og een Grav . Kan du undgaae det , saa laan ikke ; indflrcenk dig heller , saa meget du kan Ja , bed heller et christeligt <b>Hjerte</b> om en Understøttelse , for du stal gaae hen og laane . Laan gjor Lognere og letsindige Mennesker . Naar Dagen kommer at man stal betale tilbage , og man ingen Penge har , undjesr man sig fur at tilstaa dette for fin Gjceldner . Af Hovmod bliver man da en Logner , tager Skade paa sin Samvittighed, synder <b>selv</b> og frister ogsaa den Anden til Synd .

<h4>1153</h4>gaaer den Formaning : Vcerer gudfrygtige ; studerer flittigt , og beder for os herhenne ! " I den samme Tone strev deres Fader til dem og fendte dem blandt Andet Luthers Katekisme i det hebraiske , groeste , latinske og tydske Sprog , Hendes Onste , at det maatte forundes hende , endnu engang at vederkvæge sig med sin Mand men hellerikke uden ham i Friheden og Hjemmet , gik ikke i Opfyldelse . Hun oode i Fangenstab 1594. Hendes sidste Bon til de om hendes Seng forsamlede Tjenere gik ud paa , at de efter hendes Dsd vilde behandle hendes Mand med fordoblet Trostab og Omhyggelighed og vcere ganske det samme for ham , som yun <b>selv</b> saa gjerne havde villet vcrre . Dette var en fyrstelig Korsdragerinde . Nu stulle vi fortcrlls om en af borgelig Stand . I en By i det sydlige Tydstland levede der en Borger , som havde faaet Værtshuset kjcerere end sit eget Huus . Aftenerne , der hos os ere ligefom stable og dannede for Familielivet , tilbragte han udenfor sit Hjem i en Kreds af ryggeslose og lystige Kammerater . Dette fandt hans <b>Kone</b> sig i med stille Sorg , uden Trcette og stsiende Optrin . Naar han kom hjem , var hun ligefuldt venlig imod ham . Hertil bad hun daglig Herren om Styrke og ncerede det Haab , at hun vaa den Maade fnarest skulde drage sin Mands <b>Hjerte</b> fra den steile Afgrund op paa den hellige Klippe . Engang , da han feent paa Natten var sammen med sine Svirebrpdre , kom Talen paa deres Koner . Han roste sin Hustrues Sagtmodighed og paastod , at hvis de nu ved Midnat gik hjem til ham og forlangte , at hun skulde lave Mad , dcekke Bordet , og bevcerte dem , saa vilde hun gjsre det uden Knur . De Andre loe ; men han blev ved Sit . Det kom til et Vceddemaal mellem dem og derpaa til et Forssg . Den trofaste <b>Hustru</b> stod op , lavede Mad og opvartede dem ved Bordet , just ikke med Glcede , men dog med Vcerdighed og sommelig Anstand . Da blev Selskabet ilde ved ; Maaltidet varede ikke lcenge ; de Fremmede gik , og Manden blev alens tilbage . Slagen af sin Kones tjenende <b>Kjoerlighed</b> , fpurgte han hende , hvor hun fik Kraft fra til faadan Troskab . Hun svarede ham : „ Dit Liv er et Liv i Verden ; Verden forgaaer , og dens Lyst ; men bag Verden ligger der en ufalig Evighed for alle haardnakkede Foragtere af Gud , fslgelig ogsaa for dig , saaledes som du hidtil har levet . Med den Elendighed for <b>Oie</b> , som du gaaer imsde , foler jeg Trang til at gjore dig de faa Aar , fom du endnu har tilbage at leve i her paa Jorden eller som jeg endnu stal leve med dig , saa venlige og behagelige som muligt . Dette er Grunden til at jeg smer dig i Alt , og Herren giver mig Kraft dertil . " Dette gjorde Udflaget . Dette var den sidste Aften , Manden havde

<h4>1257</h4>Naar du gjor dig noget mere fortrolig med denne herlige <b>Sandhed</b> , saa staaer du alle Bern i din Menighed noer med dit <b>Hjerte</b> . Du maa bede for dem , vaage over dem og rcrkke dem en hjcrlpsom Haand , hvor du kan . De paa sin Side staae da ogfaa dig ncrr . Hvo som imooekommer dem med et faderligt eller moderligt <b>Hjerte</b> , for den bave da ogfaa de et barnligt <b>Hjerte</b> . Nu kan vistnok Ingen raMe Alle Haanden paa famme Maade . Men i enhver Menighed gives der Huller , fom den hjcrlpende <b>Kjcerlighed</b> bor tilstoppe . I enhver Menighed gives der Born , fom have mistet den ene af sine Forceldrs eller dem begge . Hvis <b>Kjcrrlighed</b> haus disse iscrr Krav paa ? Paa deres , som Gnd ikke bar stjcenket Born . I det ene Huns er der ingen Born , i det andet ingen Forceldre . Hvad er da klarere end at Gnd Herren vil udfylde Mangelen af den ene Deel med den anden ? Men har en Familie med Trostab taget sig af forcrlorelose Born , saa behsver den vist ikke i Alderdommen at klage over Eensomhed og Tomhed . Erfaringen viser noksom, at der ikke er taknemmeligere Menneffer til end forcrldrelsse Born , som have gjenfundet Fader og <b>Moder</b> . Men ville I virkelig tags et fremmed Barn til Eder som jert eget , faa tager det i saa ung en Alder som muligt . Dets Opdragelse vil da falde Eder langt lettere ; det slntter sig da langt inderligere til Eder , Dg I blive virkelig Fader og <b>Moder</b> for det . Halvvoxue Born blive ofte bestandigt fremmede i Huset , om man end behandler dem med aldrig saa megen <b>Kjcrrlighed</b> og Trostab . Vel pleie Mgtefolk <b>selv</b> efter at veers komne temmelig langt ud i Aarene fremdeles at leve i det Haab , at Gud nok endnu vil velsigne dem med Afkom , og de opsoette derfor fra Tid til anden et saadant Skridt . Deres Haab kan vistnok gaae i Opfyldelse ; men Gud vil dog snarest vise Barmhjertighed mod dem , som <b>selv</b> dave viist Barmhjertighed . For ikke trenge siden Horte jeg gammel barnlss <b>Kone</b> beklage sig og sige : „ Jeg og min ' alig Mand Have begaaet en stor Forsommelse . Vi burde allerede for femogtyve Aar siden have taget et stakkels forcrldrelost Barn til os , Da vi indsaae vor Feil , var det wr seent . Da var vi for gamle til at opdrage et lidet Barn , og havde vi taget et storre til os , vilde det strar have gjort sig Tanker om det Lille , vi eiede ; det kunde let haoe elsket vor Formue lwiere end os . Jeg mcrrker godt , at alle de , som i senere Tid have villet slutte sig til os , have vceret ledede af et eller andet Bihensyn , og jeg fsler ret tungt , hvor een , om jeg staaer <b>selv</b> i min ncermeste Kreds . " Her tan mau med <b>Sandhed</b> sige : „ Hvad du awr . det gjor snart ! " Og da give Naadens Gud dig den samme Glcede af et faa-

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011062212001'target=_>Landstad, M.B., 1880, Kirkesalmebog</a><h4>513</h4>melse skjcrnke ? Hvor maa jeg da Vcere saa inderlig glad , At mgen Sorg mig kan krcenke ! 6. Herligheds Konge , af Miflund du Mcmddom vil tåge ! Hjertet af <b>Kjcerlighed</b> fuldt jeg dig giver tilbage . Herre ! du er Den , zeg alene har kjcer , Evig jeg Synd vil forsage . 7. O min Immanuel , kom i mit <b>Hjerte</b> at fsdes ! Kom du , min Frelser ! thi uden dig maa jeg forstodes . Lev udi mig , Gjsr mig til Et nu med dig , Gud signe Dagen , vi msdes ! 8. Jesus , til dig vil jeg Hjertet , saa hsitjeg kan , hcrve , Lad mig din Indest at vinde kun stunde og strceve ! Tag mig i Tugt , Giv mig Omvendelsens Frugt : Evig for dig kun at leve ! Som : Af Hsiheden oprunden er . 136. Os er idag en Frelser fsd , Guds Salvede i vores Kjsd , En Herre , til Guds 2 Ere ; Nu er han fsd i Davids Stad , Den Son , som Englene tilbad , Velsignet evig vcere ! MinSjcrl , Kjend vel Denne Naade , Fri fra Vande , Mcet dit <b>Die</b> Ved Opgangen af det Hsie ! Som : Paa Gud alene . 137. Fra Fjord og Fjcere , Fra Fjeld og dyben Dal Et : 3 Ere vcere ! Idag gjenlyde stal . Fra Kirketaarne I Fryds Basuner stsd For Guds Enbaarne , Som er idag os fsd ; Vi var for > lorne , Nu er vi frelst af Nsd ! 2. Til Kirken samle Sig fra hver Gaard og Grend De Unge , Gamle Af <b>Kvinder</b> og af Mcend . Vi snske eder Saa glad en Julefest . Guds Riges Glceder , Guds Fred . i Jesu bedft , Hos hver , som grcrder , Vor Herre <b>selv</b> vcer ' Giest ! 3. Guds store Under ! Vi have Engle hyst , Krist hos os blunder , Guds Fred paa Jord er

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014082608023'target=_>James, John Angell, 1872, unge Kvinde</a><h4>109</h4>„ Da blev jeg hende var , ei fjern , Oss prydet , som en drsmmende jeg saa , Med alt , hvad Jord og Himmel skjsnnest har , At gjsre hende elskelig ! Hun kom Fort af sin Himmelfaders milde Haand Og ledet af hans Rsst ; Hvor yndig Gangen ! <b>Blikket</b> himmelsk var , I hver en Mine Adel , <b>Kjoerlighed</b> ! Af Fryd henreven hsit jeg da udbrod : Fuldbyrdet er , Algode , milde Skaber , nu dit Ord , Som alle gode Gaver deler ud ! Dog bedst og stjsnnest denne ' , det er Kjod Af mig og Ven af mine Ben : jeg ser Mig <b>selv</b> ! Hun <b>Kvinde</b> hede ; thi hnn blev Af Manden tagen . Saa forlade han Sin Fader , <b>Moder</b> , og de blive da Ikkun et Kjsd , et <b>Hjerte</b> og en Sjcel ! "

<h4>220</h4>HM vigtigt er det at ihukomme , at hvosomhelst der end kan hjcelpe eder , og hvilken Understøttelse I end udenfra kunne erholde til at nddanne eders Sjcel og eders <b>Hjerte</b> , saa er det dog paa eder <b>selv</b> , at dette allermest beror . Treeder ud i Livet med en dyb Forvisning om den store Vigtighed af Selverkendelse og Selvbeherskelse . Lader Grundvolden for alle eders Dyder ei , blive nogen anden end en <b>sand</b> Gudsfrygt , en Hjertets Gudsfrygt, en Gudsfrygt , fom bcstaar i Anger , Tro paa Kristus, <b>Kjcerlighed</b> til Gud samt et helligt og himmelsk Sindelag! Ingen Karakter er vel udviklet , fast , stjM eller velsignelsebringende uden paa denne Betingelse . Udvikler de Sjallens Egenskaber , som bedst scette eder istand til at opfylde eders Kald som Mandens Medhjcelp og Selskab . Uddanner eders Forstand , vorer til i ' Kundstab ; thi hvorledes skulde en uvidende <b>Kvinde</b> kunne udgjD ' re et passende Selskab for en dannet Mand ? Lcegger Vind paa et eftertænksomt og fornuftigt Sind . Ser dybere end blot til Overfladen af Tingene ; stuer hen over deres ncervcerende Udseende til deres fremtidige FLlger . Loerer at beherske eders Følelser , eders Ord , eders Handlinger. Vcerer i Særdeleshed opmærksomme Paa eders Lune , og ftger saa meget som muligt at saa det i eders Magt ; flere <b>Kvinder</b> have maaske gjort sig <b>selv</b> og andre ulykkelige mere ved Forsømmelse heraf end ved nogen anden Uvane . Efterstræber altid Idmyghed i eders Sind og Mildhed i eders Opftrsel . De cre i Særdeleshed <b>Kvindens</b> Dyder og scette hende istand til ret at opfylde sit Kald , langt mere end den stolte , befalende og haarde Handlemaade , som indeholder en fnldkommen Miskjendelse af Hemmeligheden i <b>Kvindens</b> Magt . NFisomhed og Taalmodighed , Selvfornegtelse og Underdanighed, Klogskab og Takt ere de Dyder , som bM udsaaes i Ungdommens Vaar , forat deres rige Frugter man kunne indhostes i modnere Aar . Hjertelig Velvilje , venligt Sindelag , se der , hvad I i Scerdeleshed bFr efterstræbe, fremfor alt Me . Tln disfe ere de Dyder , fom pryde en <b>Husmoder</b> i Hjemmet og en Mgtefcelle ved Mandens Side .

<h4>323</h4>Dette giver dobbelt Sjelsstyrke og Forvisning under Vandringen. Det os sterke ikke blot iog for os <b>selv</b> , men ogsall og i Særdeleshed i Herren og i hans Styrkes Kraft . Hvilket Mod skjenker ikke paa <b>Kjcerlighedens</b> Vei en Betragtning saadan som denne : „ Jeg ved , at jeg med rette Midler arbeider for et godt Formaal ; jeg vil gaa fremad i Herrens Kraft ! " Unge <b>Kvinder</b> , ogsaa i eders übemerkede Kreds , ved eders svage Vestrcebclser for at det gode skal en troende BMnens Aand , som i <b>Sandhed</b> cr den eneste , i hvilken alt burde foretages , bringe Englenes Gud til eders Hjelp og Herrernes Herre til eders Vistand . Vedbliver i den Bevidsthed , at I ret , og i den Tro , at den almcegtige gaar ved eders Side . Det betegner ikke Stolthed og Selvbedrag , men knn den Følelse as sin Evne , som enhver bM ncere . at sige : „ Jeg ved , og jeg Mer , at jeg er noget og kun udrette noget ; jeg behpver ikke at vere et Nul , et intet , thi Gud <b>selv</b> har ikke bestemt mig til intet . Jeg har Sjel og <b>Hjerte</b> , Vilje og Tunge , og med disse Gaver kan jeg noget for Gud og mine Medmennesker . Andre af min Alder og mit Kjs ' n , svage og ringe ligesom jeg , have dog udrettet noget , det kan ogsaa jeg og med Guds Hjelp og Velsignelse vil jeg det ogsaa . " Ja , du har Ret , dette er ganske <b>sandt</b> . Dette er Selvkundskab og en ret Agtelse for dig felv . Rer saadcmne Tanker , handle efter dem , og du stal udrette uoget . I en saadan Sindstilstand maa du skride til det vigtige Verk .

<h4>336</h4>Virkekredses Ensomhed , og Übemerkethed forhindrer alt dette ; intet nysgjerrig ! eller vanhelligt <b>Blik</b> kan der fFlge den unge <b>Kvinde</b> , hun er ei udsat for intetsigende Artigheder eller nogen vcemmelig Dyrkelse ; hun er der alene med Sorgen , hun har kun Gud og sin Samvittighed til Vidner . Hvad er vel , sammenlignet med en saaledes beskæftiget <b>Kvinde</b> , den skjønneste , den eleganteste <b>Dame</b> , glimrende blandt Verdens Daarstaber og Tant , beundret af mange , misundt af flere ? Hvad er vel al den Dyrkelse, som Smigerens Tunge hvisker i den ftjMnes Arc , sammenlignet med den fattiges taknemlige og inderlige Velsignelser over den Barmhjertighedsftster , som paa denne Maade „ fyldte Enkens <b>Hjerte</b> med Gloede ! " Venskabsforbindelser stjamte os ogsaa en Anledning til at vcere nyttige , som ikke t > M oversees . Maaske du i Ungdommens tankeløse Dage indgik Venskab med nogen ligesaa tankelss som du <b>selv</b> , men du er sideu vaagnet til dyb og alvorlig Besindelse . Du er ved Guds Naade gaaet over fra til Livet , men dine Venner ere endnu i Verden og vandre fjernt fra det Liv , som er af Gud . Se da her den bedste , den mest passende . Anledning til at vise din Nidkjcerbed for Gud , til ved ethvert Middel , fom Hengivenhed og Forstand lcegger for Haanden , at ftge at fp ' re , din jordiske Ven til din himmelske . Hvilken Mre , hvilken Salighed vilde det ikke vcere for dig at blive Midlet til din Vens Omuendelfe og Frelfe ! Hvor dybt og inderligt vilde ikke eders Venskab blive-fra den Stund , da Venfkabsbaandet fordobledes og helligedes ved Religionens Vaand ! Hvor frydefuld vilde eders Samliv , hvor ftd eders Forening vcere ! En Pagt vilde sluttes mellem eder , en Pagt , som vilde fLlge eder til Himmelen ; thi evigt er alt det Venskab , som hviler , paa Gudsfrygt . Tag da med dig , hvis du kan , til dette lykkelige Land dit Hjertets Ven for der at fornye , fuldkomme og fortfcette den Forening , du knyttede paa Jorden , og saaledes bevidne <b>Sandheden</b> af den Tanke , at det inderligste og lykkeligste Venskab , som begynder herneden , er blot Knoppen , og neppe det , til den Blomst , som skal leve i Himmelen for evig .

<h4>342</h4>Hustru , som endnu ikke er bleven foerdig med sine Velgjp'renh eds arbeid er eller endnu ei er kommen hjem fra et MFde eller en Forening . Ikke heller bidrager det synderlig til hans Lykke , nåar han kommer hjem om Aftenen , ndmattet as Dagens MM og Vefvcer , og fMr Trang til en Mgtefcelles opmuntrende Selskab , da istedet at maatte sidde lange Timer i uhyggelig Ensomhed , fordi hun er fravcerende paa en Sammenkomst . Det er ikke paa denne Maade , at en gift <b>Kvinde</b> befordrer den fcelles Lykke eller vinder sin Mands <b>Hjerte</b> for de hellige Ting . Det bringer aldrig noget godt at tjene Gud saaledes , at man forsømmer sine huslige Pligter . En forftmt Mgtefcelle og vanskøttede Bjsrn er et sørgeligt Bevis paa nogle <b>Kvinders</b> urigtige Opfatning af sin religiøse Pligt , og dette Bevis fremføres ikke sjelden til Men for den Religion , de ville tjene . Af faadllnne Exempler benytte ogfaa de sig , som vel kunne gjFre meget , men nu ei ville gjMe noget for Religionen , og just dermed retfcerdiggjMe de sin Lunkenhed og Selvifkhed ; og i <b>Sandhed</b> , jeg er ilde tilmode ved , at de skulle sinde saadanne Exempler at fremfare . En gift <b>Kvinde</b> kan imidlertid , ogsaa uden at opofre sine huslige Pligter , have titstrcekkelrg Tid og Frihed til at tjene Religionens og Menneskehedens Sag ; nogle gjM det ogsaa , de nemlig , som med Orden , Flid og Raskhed indrette sine huslige Arbeider derefter . Salomos Beskrivelfe af den dydige <b>Kvinde</b> indbefatter begge Fortjenester . „ Hun holder <b>Die</b> med sit Hnses Veie . Hendes Mands <b>Hjerte</b> lider paa hende , og Vinding skal ei fattes . Hendes Sonner staa op og Prise hende lykkelig . Hun giver sit Hus BrFd og sine Piger deres bestemte Del . Ladheds BrFd spiser hun ikke . " Her se vi Orden , Omtanke , Vindskibelighed og Beregning ; og desuagtet , med alt dette er forenet : „ Sin Haand aabner hun til den elendige , og sine Hcender udrcekker hun til den fattige . <b>Kjcerligheds</b> Lov er Paa hendes Tunge . " Se der den gode <b>Husmoder</b> , som finder Tid ogsaa til Kjcerlighedsgjerninger udenfor Hufet , uden at forsMme Vindskibelighedens Pligter i Hjemmet . Naar Mandens Hygge ikke forsømmes , naar han ikke klager

<h4>664</h4>Bevceggrund i og for Menneskenes Handlinger . „ Med Mennestehaand trak jeg dem , " sagde Gud , „ med Kærligheds Reb . " Heri ligger den sande Hemmelighed baaoc af Menneskets naturlige Sjcelckultur og af Evangeliets himmelske Magt . Menneskenaturen er snarere dannet til at styres og ledes af <b>Kjcerlighed</b> , at drages med <b>Kjcerlighedens</b> og ømhedens Baand , end til at flcebes fremad i Strenghedens Kjoeder . Og Kvindehjertet er skabt til at elske : <b>Kjcerligheden</b> skjoenkes mere mild , M og tvingende af <b>Kvinden</b> end as Manden . Den hende mere taalmodig og vis og derfor mere indflydelsesrig . Hendes Ord ere mere mildnende , hendes Gloede mere vindende , hendes Pande mere tvingende , netop fordi den er mindre truende og frastødende . Den speede Plante , hun har at vogte , aabner lettere sin Blomsterkalk for Lyset i hendes Blikke . Derfor er ogfaa en Moders Magt over sit spåede Barn den Drste moralske Magt paa Jorden . Og der ligger ei synderlig Overdrivelse i det Udtryk , at „ hun , som gynger Vuggen , styrer Verden . " <b>Sandheden</b> deraf vil blive tydelig af hvad fom ftlger . Moderen har at vaage over Karakteren , medens den endnu af de ftrstc Indtryk erholder sit Prceg og sin Form . De tidligste Mringer hos Barnet af Tanke , Melse , Vilje og Samvittighed fremtrcede under hendes Hun har ikke blot at gjMe med Legemet i dets Barndom , men ogfaa med Sjcelen i dens ftrste Udvikling . Bande Sind og <b>Hjerte</b> ere i hendes Haand , naar de gjMe sine ftrste Anstrengelser til godt eller ondt . Af hende lcere VMnene at stamme de fprste Ord , uuder hendes Ledelse danne de sig de ftrstc Begreber . De ere noesten bestandig i hendes Selskab , og nden at de <b>selv</b> vide det , eller hun merker det , bestemmes as hende deres gode eller slette Retning . Hun er deres ftrste Erempel , det ftrstc Mynster , hvorefter de rette sig ; Ret og Uret fe de allerførst i hendes Handlemaade. De ere bestandig Vidner til de Fortrin eller Feil , som vise sig i hendes Ord , Gemyt og Handlinger . Uden at vcere sig det bevidst , opdrager hnn dem faaledes , ikke blot ved den Undervisning , hun meddeler dem , men ogsaa

<h4>675</h4>Men er det vel at vente , eller er det endog muligt , at nogen kan nndervife med Held i en Religion , fom man ei felv kjender og udMer ; kan nogen lede sit Barn til den Frelser , som han <b>selv</b> ei kjender ? Derfor maa en Moders <b>Hjerte</b> fremfor alt vcere opfyldt af en <b>sand</b> Gudsfrygt , hvis hun deri stal kunne nndervife sit Barn . Kan hun vel uden dette have Lyst til at undervise , <b>Hjerte</b> til at bede , Ret til at haabe ? <b>Moder</b> , kan du tcenkc dig noget cedlere , liMre , vigtigere Kald end at aabne det undrende Barnesind for den ftrste Tanke om Gud ? At lede dit eget Barn til det Gudebarn , som ftdtes i Bethlehem , var sine Forceldre underdanig , vorte op til en Frelser , som sagde : „ Lader de smaa BMn komme til mig , og forhindrer dem ikke ; thi Guds Rige lftre faadavne til , " tog dem i Favn og velsignede dem , og fom endelig led Korfets T ) Dd for at give dem et evigt Liv ? Eller at tale for det om Himmelen, hvor Gud og de hellige Engle bo ? O , at se det fyrste hellig spørgende <b>Blik</b> , den ftrste Tåare i troende Ainc , at lMe det ftrste SpFrgsmaal eller den ftrste BM fra fpcede Bprnelceber ! Hvor mangen <b>Kvindes</b> <b>Hjerte</b> har ei ved en saadan Anledning strammet over af Glcede og bragt hende til at synke ned paa sine Knce for at udgyde sine Følelser i VM over sit Barn , som undrende hcevede

<h4>764</h4>Haand hjelper , at tMme Sorgens Boeger . Nuvel , tag det og det ligesom den største og helligste Korsdrager , der nogensinde gik gjennem Taarernes Dal , med de Ord : „ Skal jeg ikke drikke den Kalk , som min Fader gav mig ? " og den Tid vil snart komme , da han , som elsker dig hpiere end dn elsker dig <b>selv</b> , skal aftprre alle Taarer fra dine Ame . Til dem atter , som ved Guds Naade ftge at danne sig efter det MsMster , som vi i dette Kapitel have fremstillet, og som ere i de Omstændigheder , at de kunne gjpre dette , til dem Mskede jeg at sige : Kaster Mildhedens , ydmyghedens og Blufærdighedens SIM over alle disse stMne Former af Virksomhed og Kraft . Glemmer fremfor alt ikke det elskelige Trcek : „ <b>Kjcerligheds</b> Lov er paa hendes Tunge . " Forener <b>kvindelig</b> Nmhed og Blidhed med al denne mandige Kraft i Udførelse af Livets Pligter . Hvilke eders Egenskaber end maa vcere , ncerer fremfor alt <b>Kvinder</b> ; og naar I , i Egenskab af Mgtefwlle , <b>Moder</b> og Husbestyrerinde , udMe den Magt og Myndighed , fom eder tilkommer , saa glemmer dog ikke , at der sindes en i Familien , jeg mener eders Mand , hvis Myndighed er endnu stirre end eders , og at det er Paa engang eders Pligt og skal blive eders Lykke med Amlzed og <b>Kjcerlighed</b> , ei med slavisk Underdanighed , at underkaste eder ham . Nnge <b>Kvinder</b> , jeg lcegger eder paa <b>Hjerte</b> at gjFre eder fortrolige med dette skjenne Sted i den hellige Skrift . Gjorer det til Gjenstand for eders stadige Betragtning ; meget , scerdeles meget kunne I lcere deraf . I ville der se , at Gudsfrygt er den sikreste og fasteste Grundvold for al <b>kvindelig</b> Dyd , og at den langtfra at ophceve eller vore jordiske Pligter for ncer , tvertimod overalt , hvor den er fand og levende , skjcenker et klarere <b>Blik</b> paa dem og en stsrre Evne til at opfylde dem . Gudsfrygt er nyttig til alle Ting og befordrer alle lovlige Bestræbelser . Der findes ei et eneste cedelt Karaktertrak , som ei Religionen endnu mere hcever og forwoler , ingen jordist Interesse , forudsat at den er tilladelig , som den ei merkelig befordrer . Lader ikke dens Fiender indvirke Paa eder med sine urigtige Forestillinger , ligefom om Gudsfrygt var uforenelig

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009031312001'target=_>Lous, M.C. Theodor, 1865, De ti Bud</a><h4>240</h4>Huus , at virke til vort <b>Hjertes</b> og vort hele Vcrsens Helligelse, da vil Herren <b>selv</b> lcrgge sin Velsignelse til vor Gjerning og udrette i os , at vi ved hans Naade vinde Kraft til allerede paa Hviledagen at begynde og gjennem Ugens svrige Dage fortscette som fornyede i vor Einds Aand at ferdes i <b>sand</b> Christelighed blandt vore Brpdre , mcd Sagtmodighed, Taalmodighed , <b>Kjcerlighed</b> og Forligelighed , stottende de Vaklende , strcebende at reise de Faldne , troftende de Bedrsvede , opmuntrende de Forsagte , kort i Alt og blandt Alle i en <b>sand</b> christelig Vandel at aabenbare den udvortcs Gudsdyrkelses bedste og sijonneste Deel i de Dyder og sande christelige gode Gjerninger , som vi , drevne af Guds Aand , udove i det daglige Liv , lac . 1. 23 , 27. Frugten for os af en saaoan Helligclse af Hviledagen formedelst Guds Ord , vil da blive en klarere Indsigt i Guds Rige , Kraft til en from Vandel og deraf selgende stprre Tillid og Fortrøstning til Herren . Det er en Selvfølge , at den Christne , der som Familiefader har Tjenere og Born , eller som Overordnet i andre Livsforholde har Underordnede , der stylde ham Lydighed , med Hensyn til det tredie Bud har den Pligt , ikke alene ikke at hindre disse sine Undergivne i at benytte Hviledagen efter dens Bestemmelse ; men endog meget mere ved eget Exempel , ved Veiledning , Raad og Formaning, om det skulde fornsdiges , at lndrage til , at den i <b>Sandhed</b> kan blive ogsaa dem en Helligdag , 3 : en Helligelsesdag. Medchristne ! Hvor uendelig naadig har dog ikke Herren vceret mod os , at han forundte os at fsdes og leve i et christent Land , hvor Indbydelser til Herren vor Gud lyde til os ved hvert Skridt , vi gjsre paa Livets Vei . Hvor vi vende vort <b>Blik</b> , <b>moder</b> dct Skole ved Skole , Kirke ved Kirke , hvor Guds <b>Sandheds</b> Ord lceres og prcedikes puurt og reent , hans Eacramenter forvaltes uforfalsket efter Herrens egen Indstiftelse , saa Ingen kan vcere ganske fremmed for Herrens Naadesforanstaltninger til Synderes Frelfe , hver E sndag og Helligdag kalder Klokkeklangen fra Herrens Templer, og indbyder hver Enkelt at trcede ind i Herrens Huus for at opbygges , at twstes og styrkes ved det Ord , der er en Guds Kraft til Ealiggjorelfe for hver den , fom troer . Men lader os dog ikke glemme , mine Venner , at alt dette dog ikke gavner os noget , saafremt vi ikke ville lytte til

<h4>277</h4>at forstaae , at der ingen Forskjel maa vcere i <b>Kjcrrlighedens</b> Styrke og Inderlighet ) ; <b>selv</b> Gall . 6 , 10. , hvor det heder :  Derfor , eftersom vi have Leilighet ) , lader os gjme det Gode mod sllle ; men mest mod Troens Egne " , viser os , at Religionens, og saaledes ogsaa Mgtestabets og Stcegtskabets hellige Baand maa stille os enkelte Personer ncermere end Andre, gjsre dem dyrebarere for vort <b>Hjerte</b> og gjsre flere og sterre Opoffrelser for dem ikke alene mulige , men endog til Pligt for os . Tilsidst maa vor Mcnnestekjcrrlighed verre 4 ) viis i sine Åttringer , 3 : at vi , i vore Bestrebelser for at hjcelpe Andre og fremme deres Vel , tåger Hensyn til de Krafter og Evner , vi have at virke med , saa vi ikke af misforstaaet Iver for at gjsre ret Meget , maaskee sstte os ud af Stand til at virke Noget , at vi gjsre det Ene uden at forssmme det Andet ; fremdeles , at vi ikke gavne Een , f aa ledes at eller vcd at vi stade en Anden , at vi fcrtte det vigtigere Gode foran det mindre vigtige , saaledes Sjelens Vel foran det sandselige og timelig / , at man veier de Enkeltes Drre eller mindre Trang eller Vcrrdighed ; endelig , at man ikke glemmer dem , der staae os ncermere , for dem , der have mindre Krav til os , saaledes at <b>Hustru</b> og Born gaae foran Elcegt , Venner og Velgjorere , disse igjen foran andre Landsmcrnd og Troesforvandte og disse igjen foran Fremmede , dog aldrig glemmende , at de Alle ere vore Brodre og vor Naste ; kort sagt , at man forener Klogstab med sin Kjærlighet», saa man ikke , trods sin bedste Villie , kommer til at stade istedetfor at gavne , cfr . Rom . 16 , 19. Kjærligste Gud og Fader ! Du , hvis <b>Kjcrrlighed</b> hver Dag er ny over os ; du , som i din uendelige <b>Kjcrrlighed</b> offrede din egen.eenbaarne Ssn for os , og ved din trofaste Helligaand utrettelig arbeider for vor Saliggjsrelse ; men ogsaa fordrer , at vi stulle elske dig igjen , og vise denne vor <b>Kjcerlighed</b> derved , at vi elske vore Brydre , ak , Herre , du seer det kun altfor vel , hvor ftrgeligt det staaer til med <b>Kjcerligheden</b> iblandt os ; det falder saa vansteligt at fele <b>sand</b> ' og oprigtig Glcede ved Ncestens Lykke og Velfard , det falder saa tungt for Kjsd og Blod at elske sin Fiende , at gjengjcelde Had med <b>Kjcrrlighed</b> , Vrede med Sagtmodighed , og dog vide vi , dette maa stee , saafremt vi stulle blive dine sande Disciple , O , hjcelp os da , kjerre Herre ! ved din trofaste

<h4>285</h4>Andre ; thi gode Born blive ogsaa gode Christne og gode Borgere . Den fyrste Fordring , Lovens anden Tavle gjor til os , er altsaa , at vi stulle hcedre vore Forceldre , d . e . i vort ganske <b>Hjerte</b> og ved vort hele Forhold i Livet vise , at vi erkjende dem som vore fsr ^ e Velgjørere , givne af Gud , og at vi desaarsag iscerdeleshed agte , elske , tjene og « ere dem . Vi maa her lcegge Merke til , at det heder ikke blot , vi skulle elske dem , men at vi skulle crre , hcedre dem , hvorved er lagt os paa <b>Hjerte</b> , at vi stulle formedelst deres Verdighet ) stille dem hsiere end os <b>selv</b> , ansee dem som Personer , der af Gud <b>selv</b> ere beklcedte med Majestcet og Herredom , cfr . 3 Moseb . ! 9 , 3. Luther siger derom i sin store Catechismus:  Denne Fader- og Moderstano har Gud i Besynderlighed eret fremfor alle timelige Forhold , som ere under den , idet han ikke alene simpelthen byder at elske Foreldrene, men at hedre dem . Thi mod Bwdre , Sestre og Nesten i Almindelighed befaler han intet Hsiere , end at elste dem , saa at han ligesom affondrer og adstiller Fader og <b>Moder</b> fra alle andre Personer paa Jorden og setter dem ved Siden af sig <b>selv</b> . Thi det er en langt hpiere Ting at hcedre end at elske ; hint indbefatter ikke alene <b>Kjcerligheden</b> i sig , men tillige en vis dyb Idmyghed og LErefrygt , fom for en forborgen Majestcet " . Sporge vi nu : hvorpaa grunder sig den hoie Verdighed, den Majestaet , hvormed Herren <b>selv</b> har vekledt Faderog Moderstanden ? da svare vi : l ) Fordi Gud <b>selv</b> har verdiget at betroe unge Sjele til deres Varetcegt og Omsorg, har sat dem til , i hans Navn , at opholde , opdrage , lede , styre , formane og troste dem , saa at Herren <b>selv</b> just ved Foreldrene daglig udpser sin Velsignelse over Bornene , saa at Foreldrene altid staae som Mellemmend mellem Gud og Byrnene , som Guds synlige Representanter for disse . 2 ) Fordi ved dem og deres <b>Kjcerlighed</b> til deres Born stilles os for <b>Die</b> det skMneste Billede af Guds Spns opoffrende <b>Kjcrrlighed</b> , der ikke ftgte sit eget , men vort Vel . Den troe <b>Moder</b> opofsrer villig sine Krcrfter , sin Roligheo uden at trettes . Ja <b>selv</b> hos Dyrene har Ekaberen indplantet en beundringsverdig Moderkjcerlighed , saa at <b>selv</b> de stundom kunne aabenbare , at et Aandepust fra Herren har nedladt sig i deres Legemer .

<h4>468</h4>bliver den syndig , Eph . 4 , 26. , Matth 5,23 - 25. 3 ) Misundelse, der ikke under Ncesten noget Godt , men heller glceder sig ved og gjerne ønsker at fremme hans Ulykke , lac . 3 , 16. 4 ) Übarmhjertighet » , der ikke rores ved Ncestens Md og Jammer , lac . 2 , 13. Spsrge ri nu : hvorledes stulle vi bedft kunne undgaae paa nogen af de ovenfor omtalte Maader at forsynde os mod vor Ncrste , da er Svaret : Ved at srygte og elske Gud , saa vi vogte os for at krcenke og stade hans stjsnneste og herligste Skabning paa Jorden ; ved at beflitte os paa en <b>saud</b> og virksom Kjcrrlighcd mod Nersten ( Rom , 13 , 10 ) , og ved at vi stedse tcrnke os i Ncestens Sted og sp ^ rgc os <b>selv</b> : hvorledes vilde nu du , om du var i hans Sted , helst enste dig imsdekommet ? Medchristne ! Vi gyse , naar vi hyre tale om Mord og Drab ; vart <b>Hjerte</b> krymper sig af Forferdelse , naar vi fee den nlykkelige Morder , og vi ere saa tillwielige til at mene , saa vybt ere vi dog ikke faldne . O , lader os dog ikke bedrage os <b>selv</b> , mine Venner ; lader os dog ikke glemme , det er ikke Haanden blot , men Hjertet , som gjM os til Mordere; thi Herren har jo sagt : Hvo som haoer sin Broder , er en Manddraber , og hvad hjcrlper det os vel , at Menneskene ikke kjende os som Mordere , naar Herren i Himmelen demmer os som saadanne ? Derfor , min Broder ! vil du prove dig <b>selv</b> efter dette Bud , da fee ikke blot til hvad Haanden har gjort , men vend dit <b>Blik</b> indad i Hertets inderstc Grunt , og fpstrg dig <b>selv</b> , som for Guds Aafyn : hvad er det , fom har boet og boer derinde ? Har aldrig Hadets og Hcenngjerrighedcns vilde Flamme drevet Broderkjcrrligheden ud ? Har aldrig Vredens Kulde stukket <b>Kjcerlighedens</b> Ild ? Har aldrig Bitterhed , Hiosigheo , Ekjcrldsord saaret og bedrsvct din Ncrstes <b>Hjerte</b> ? Og hvo blandt os Alle maa ikke da beskjemmet vende <b>Blikket</b> mod Jorden og beljende: Ak , Herre ! der fattes mig saameget ogsaa i dette Stykke i at kunne bestaae for dig ! Ja , i Eandhed mine Venner , Adams Eced boer i os Alle , og vi trcrnge Alle til at bede : Herre vor Gud ! tcrnd du felv <b>Kjcerlighedens</b> Flamme i vore Hjerter , at den maa drive Hat » og Fiendstab , Haardhed og Bitterhed ud af vort Vcrsen ; lad din <b>Kjcerlighed</b> varme os saa , at vi oprigtig lcere at elske ham , der lod sit Hjerteblod flyde for vore Synder og felv paa Korfet

<h4>585</h4>i dette , det vigtigfte af Alle , her maa Religion og Bon vare vor daglige Gjast , og fremfor Alt bede vi daglig , at Herren ret vil aabne vor Forstand og vort <b>Hjerte</b> , saa vi ved hans Hjcelp og Naade kunne blive dygtige til at fore et crgtestabeligt Liv , over hvilket hans Velbehag kan hvile . <b>Hustru</b> , Born , Familie , Huus og Hjem , hvormeget skjsnt , hvormeget elsteligt ligger der ikke indbefattet i disse Ord , mine Venner ! de indbefatte i sig Forestillingen om de reneste , de Edleste , de stjsnnestt ' Glceder , og alt dette har Herren vor Gud beredet os i den hellige LEgtestand , som han allerede indstiftede i Paradiset og gjorde til en Velsignelses, en Helligelftlis Ctand for vor hele Uddannelse paa denne Jord ; o hjcelp os da . Herre , at vi ogsaa stedse og altid maa agte denne Stand og bruge den efter din kjcerlige Hensigt , at dm Naade fremdeles kan hvile derover ; lad <b>sand</b> <b>Kjcerlighed</b> boe i vore Huse , lad Nsisomhed , Tarvelighed og Orden smykke vore Boliger ; men fremfor Alt lad Gudsfrygt hellige vore Hjerter , at Fred og Enighed kan boe i Hytte som i Borg , at Bsrnene maa opdrages i Tugt og Wrbarhed, dig til Lov og Priis , dem <b>selv</b> til Fred og Salighed . Ia styrk os . Herre , at vi , <b>selv</b> rensede ved din Aand , kunne fyre Reenhedens , Hellighedens og <b>Kjarlighedens</b> Aand ind i vore Huse og vore Familier , at de kunne blive din Helligaands Templer , der , reiste paa Jorden , vege mod Himlen ng i kjarlig Forening fore Mand , <b>Hustru</b> og Born ind til din Himmelborgs « forkrenkelige Teier , Det skee . Herre , for Jesu Christi Skyld . Amen .

<h4>809</h4>see hans Velfcrrd fremmes , bedroves over hans Tab og Skade , og for Herrens Skyld af <b>sand</b> Broderkjcerlighed gjore hvad vi formane , for at hjcrlpe til at han kan ikke blot beholde og nyde hvad Herren har tildeelt ham , men endog om mnligt see det voxe og formeres . Men stal dette kunne ster , stal det blive os muligt endog blot med Oprigtighed at strcebe til at fyldestgjore denne Fordring , da er det ogsaa en nsdvendig Betingelse , at vi have et saadant Herredsmme over den i vort Vcrsen nedlagte Begjcrrevne , at den ikke gaacr i det blot sandselig Behageliges , det verdslig Nyttiges Tjeneste , men med <b>sand</b> christelig Frihed higer mod det Guddommelige , det Gud Velbehagelige , saa at vi i Alt hvad vi begjcrre forst og fremst have hans Villies Velbehagelighed for <b>Die</b> , og vort <b>Hjertes</b> Lyst altid staaer til Herren ; Ps . 37 , 4. Det Menneske , som ved Herrens Naade har vundet saadant Herredsmme over sit eget <b>Hjerte</b> , har , som vi i det Foregaaende stildrede , naaet frem til den fande christelige Frihed , ikke lcrngere trcellende under Begjcerlighedcrnes Aag , bestemmer han med Frihed over sin Villies Ittringer og den sonlige Udkaarelses Aand , som har faaet Rum i hans <b>Hjerte</b> , den <b>Kjcrrlighedens</b> Aand , der har gjort ham til et frit Guds Barn , vil vise sin Frugt i Admyghed , Sagtmodighed , Godhed og Kjerrlighed i alle hans Forhold , al hans Fcrrd mod Nersten . Den borgerlige Lovs dode Bogstav er ikke lcrngere Regelen for hans Opforsel mod Nasten, men den Broderkjcerlighed , ' som fylder hans Sjel , han fpsrger ikke , har jeg Ret til at tilegne mig dette eller hint af Ncrfiens Gods ? men , bpr jeg her benytte min Ret , kan jeg benytte den uden at stade min Nceste mere end jeg gavner mig <b>selv</b> ? Han tanker ikke paa , om han kan forsvare for Menneskene at tilfredsstille dette eller hint Onfle med Hensyn til hvad Ncesten kalder Sit ; men derimod , kan jeg elske denne Broder og dog gaae frem paa denne eller hin Maade mod ham ? vilde jeg pnste at Andre håndlede saaledes mod mig ? Kort sagt , her befales os at Aroderkjcerlighedens Aand stal gjsre sig gjcrldende i alle vore Forhold mod Nersten , saa vi ikke blot see hen til vort eget , men ogsaa til Andres Bedste , at derfor Aanden stal herske over Kjpdet i os og vise sin Frugt i vor hele Vandel ; men standens Frugt er <b>Kjcrrlighed</b> , Glcrde , Fred ( Gall . 5 , 22 ) . Vi ville endnu til Slutning i en Hovedsum sammenfatte disse

<h4>895</h4>dem , deels gjennem Foreldrenes gode Exempel og christelige gudfrygtige Opdragelse og Veiledning , deels paa andre umiddelbare Maader en fcerdeles Hjcelp og Kraft , til Ugesom Forcrldrene at vandre paa Herrens Veie , og saaledes fremdeles at nyde Velsignelfen af hans Naade og atter igjen at kunne forplante den over paa sine Bsrn ; medens derimod den , fom trods denne Herrens Velsignelse og den deraf fiydende Kraft og Naade , forhcerder sit <b>Hjerte</b> og viger af fra hans Veie , derved forer Forbandelsen over sig , og det saameget mere , jo flere og kraftigere Opfordringer , jo stsrre Hjcelp fra Gud og Mennesker han har havt til at gjsre bedre Valg . Heri ligger jo netoft en fcerdeles kraftig Opfordring til os Alle , til ret med Aarvaagenhed paa os <b>selv</b> at agte paa Herrens Bud , af ganske <b>Hjerte</b> at elske dem og stadig have dem for <b>Die</b> , ja uden Aflaoelse at ftrcebe til at kunne danne vort hele Liv efter dem , for derved at fore Velsignelse og Herrens Naade ikke blot over os <b>selv</b> , men <b>selv</b> over vore Bsrn og Efterkommere i tusinde Led . Ved det evige Liv forstaaes de Frelstes fuldkomne og inderlige Samfund med Gud og alle de falige Aander og al den Lykke og Salighet » , som maa verre uadstillelig fra dette Samfund , hvor man befriet fra al den Skrsbelighed og Indskrcenkethed , som herneden klcrber ved vort Vcesen , stal stue <b>Sandheden</b> , Gud <b>selv</b> , Ansigt til Ansigt , og der finde Lesningen paa alle Livets Gander , Tilfredsstillelsen af alle Sjelens Savn og Lengsler og i en evigvarende Fred og Glcroe med de hellige Engle love Gud . Al denne Velsignelse, denne Miskundhed tilsiger Herren Alle dem , som elske ham og holde hans Bud ; det bliver derfor nsdvendigt for os at vide , hvao det er at holde hans Bud . I de indledende Bemerkninger , fom forudstikkedes Gjennemgaaelsen af de enkelte Bud , blev der gjort opmcerksom paa ( Pag . 9 — 12 ) , at Loven ikke kan tilfredsstilles ved udvortes Gjerninger , men at han krcever det ganste Menneske med Legeme og Sjel , Luc . 10 , 27. , eller at den hele Lov med alle dens Fordringer sinder sin Fyldestgjsrelse i dette Ene , <b>Kjcerlighed</b> ,  <b>Kjcerlighed</b> af et reent <b>Hjerte</b> , af en god Samvittighed og af en uflrsmtet Tro " ( l Tim . 1 , 5. ) , med eet Ord , altsaa et reent <b>Hjerte</b> . Guds Bud kan altsaa alene holdes ved et Gud velbehageligt Vcefen , der har sin Rod i et reent <b>Hjerte</b> , som udvortes yttrer sig i

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111303026'target=_>Rudelbach, Andreas Gottlob, 1848, Christelig Biographie</a><h4>1029</h4>Saaledes bleve TilkmMlsespunkterne til Hedenskabet mangfoldiggjorte i den fsrste christelige Kirke ; et Net blev udkastet , hvis Storhcd og Rummclighed kun kan lignes ved det , der udkastedes af Diseiplene ved Sscn Tiberias efter Herrens Opstaudclse ( Joh . 21 ) — i hvilket , som bekjendt , den christelige Oldtid saae et Forvillede paa den store Apostoliste Folke-Dratt . <b>Kjerligheden</b> gjorde ( som altid ) opfinderisk ; man var uudtommelig i at fremstille Christendommens tiltrcekkende Kraft liig en stor himmelst Magnets , der nu strakte sig til alle Jordens Ender . Sjelens Sul og Loengsel ,  den af Naturen christelige Sjels " 6 ) Kamp efter i ^ ys og Fred ( som Tertullian udtrpkker sig ) — det llbevidfte , Mennesket i dets naturlige Tilstand , fslgelig og alle Hedninger , Forborgne , hvorved detevigeLys vidnerom sig <b>selv</b> ( som ' f . Er . naar Hedningene sagde : Gud veed det ; Gud seer det ; det lader jeg Gud raade ) — Samvittighedcn som et dobbelt Vidne baade om Menneskets personlige Ansvarlighet » og om den , der i alle Tilfcrlde stal og vil krceve ethvert Menneske til Regnstab ' ) — > det var een af de Bttragtninger , hvorved man sogte at vise , hvorlunde Christendommen laae som en stille , tilbagetrcrngt Hjerte- Rost ncer ved Enhvers <b>Hjerte</b> . Man g > k et Skridt videre : man tog ei i Vetcenkning at indromme , at , skjondt jo Philosophcrne maatte ansees som  Lognens Oldefcrdre ( Patriarcher ) " , netop fordi de i censidige og tomme Grublerier havde forvandstet det Klareste og gjort det Sikkreste tviulsomt , ^ ) alligcvel ogsaa hos dem saavelsom hos Pocterne fandtes mangt et Sandhcds Gimt ,  ei som om de <b>selv</b> havde erkjendt <b>Sandheden</b> , men fordi <b>Sandhedens</b> Kraft er saa stor , at <b>selv</b> det blinde <b>Die</b> skimter den sig fremtrcengende Guddoms-Klarhed " ; ^ ) thi det er jo det samme , som vi see i naturlige Forhold ,  at stundom i Storm , naar Himlcns og Havets Spor sammenblandes , alligevel ved en lykkelig Forvildelsc Havnen naaes , og stundom i Morket Indg , angen eller Udgangen gribes ved en blind Lykke " . ^ " ) Det var Hedningehjerter,

<h4>1640</h4>X . Men pludseligt formorkedes Himlen . Kong Frederik l V var dsd den 12 Octobcr 1730 paa Christi , sin Forlosers , Troe . Eftcrretnmgen derom bragtes forst til Gronland 1731 ; den ledsagedcs as den allerbedrovcligste Tidende . Ordren lsd ftaa , at Colonien for de store Udgivters Skpld , som forgjeves vare anvendte paa den , skulde ovhcrves ; Folkene skulde strarhjem ; vilde Egede og nogle med ham blive , saa skulde der efterlades dem eet Aars Proviant , dog kun under den Forudsoetning , at de Tilbagcvendende forst vare forsynede med det Fornodne . Egede var som ssnderknust ; hans sorrigfulre <b>Blik</b> dvcrlede paa eengang paa hans  losias " ( den aflode Konge , der vel fortjente dette Navn ) og paa den Grsnlandske Menighed , der var at forligne med et svagt Barn ;  scrrlig de Smaae , som vare dobte (omtrent 150 i Tallet ) " , siger han ,  laae mig saa ncer paa <b>Hjerte</b> , som nogensinde Born ku > ine ligge en kjer <b>Moder</b> paa <b>Hjerte</b> " .  Er det <b>sandt</b> " , kom Grsnlcendernc og spurgte , Du vil forladc os ? " Og > da han havde bejaet det , sigende , at saa var Kongens Villie , og han havde vel besluttet det , fordi han saae , at de ikke forbedredes og forfvemmedes trods den christelige Undervlislnng, svarcde de , at de , som havde sagt Kongen noget stigt , vare store Lsgnere ;  Du veed jo <b>selv</b> " , vedblcvc de ,  at vi flittig have givet Agt paa din Underviisning " ^ Imidlertid

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009072703011'target=_>Ahlfeld, Friedrich, 1863, Menneskelivet</a><h4>191</h4>det itke at see Vornene udvitle sig . Naar vi <b>selv</b> have plantet eller forcedlet et Trce , og det da bcerer de forste to eller tre Frugter , saa glcede vi os mere herover , end om Andre bragte os Kurve fulde af Frugter . Og nåar vi bemcerte, hvorledes de forste Tanter bryde frem hos Vornene , hvorledes det skjulte Liv aabenbarer sig i de forste tydelige Ord , hvor glcede vi os da ikte ! Og dog er dette Intet mod den Fryd , vi fole , nåar vi see det lille <b>Hjerte</b> oplade sig i Troen , nåar det beder sine forste Bonner og slutter sig med barnlig Tillid og <b>Kjcerlighed</b> til sin Frelser . Mange Forceldre maae beljende : „ Mine smaa Born have atter fort mig tilbage til min Gud og Frelfer . Min Tro var faa dod , den lignede kunstige Blomster uden Saft og Kraft . Da horte jeg , hvorledes Vornene bade , hvorledes de omgittes med deres Frelser og tillidsfuldt udoste deres Hjerter for ham . At , da blev jeg faa veemodig , da tcentte jeg paa det Ord : Uden I blive som Born , tomme I ingenlunde ind i Himmeriges Rige . Da gjoroe mine Born mig Gamle til et Barn igjen " . De have ogsaa vcennet mangen Gammel af med at bande , lyve , bedrage , fvire og andre Laster , ved bestandig at gjentage den eenfoldige Paamindelse : „Fader eller <b>Moder</b> , det tan Gud ikte lide " . Her er i <b>Sandhed</b> Grund not til Glcede , men ogsaa Grund not til at fore Vornene hen til Herren forn hans Gave .

<h4>408</h4>fade inde i Kornet , bede med hinanden , og i stille Andagt og Glede lyttede de til . Men da det ene af Bornene var ferdigt med at bede , sagde det til det andet : „ Horer Du , hvor smutt jeg tan bede ? " og ved disse Ord vnr det , som om de to Venner sit en Spand koldt Vand over Hovedet . Forfengelighetens Orm havde ogfaa gnavet sig ind i Bonnen; thi den staaner Intet . Det Menneske , i hvis <b>Hjerte</b> den har bidt sig fast , lerer aldrig at kjende den fande Fred og No . Hans Liv er feberagtigt ; hans Puls slaaer hurtigt , hvergang hans Torst efter A3re tan blive tilfredsstillet ; og nåar der ingen Leilighet » er dertil , eller nåar han maa staae tilbage for Andre , bliver han told og foleslos . Ak , hvor fattigt bliver dog et faadcmt Menneske ! Hvergang han forbignaes med Taushet » , hvergang han dndles , er det , forn en told Stylregn styrtede ned over ham . Og hvor meget der end istjentes ham af 3 Erens berusende Drit , bliver hans Torst dog aldrig stillet ; thi 3 Ere er som Havvand , jo mere man dritter deraf , desto mere torster man. hofferdighet » er forn en Ulv , der aldrig bliver met . <b>Sand</b> No og Tilfredshet» er tnn at finde i Gud . — Dette , tjere Lefere , er den ene Side af den Ulykke , som den forfengelige 3 Ere styrter os i . Vi blive indvortes fattige ved denne Rigdom. — Men der gives ogfaa en anden . 3 Ergjerrigheden er <b>Moder</b> til mange Synder . Hovmod var , efter vore gamle Theologers Mening , Aarfag til Djevelens Fald , og ved at indbilde vore forste Foreldre , at de vilde blive ligesom Gud , lokkede han dem til at synde . Sirach talder ( 10 , 14 ) Hofferdighet» en Begyndelfe til Synd ; Tobias siger ( 4 , 17 ) : „ Udi Hovmod er Fordervelse og megen Übestandighed " , og vor Text setter Tr ods og Avind i Forbindelse med forfengelig Mre . De ere ogfaa nogle af de forste Folger , som den har . Den , der gjor LEren til sin Gud , vil tilegne sig <b>selv</b> al Anertjendeise eg Roes . Hnn bliver told mod sine Medmennesker , og de blive hnm tilsidst tun Trin paa den Stige , som han vil heve sig op paa . Han bliver hjertelos , han har ingen <b>Kjcerlighed</b> til Andre , og han <b>moder</b> heller ingen <b>Kjerlighed</b> ; tilsidst har hnn ingen Venner mere . Sirach siger ( 10 , 7 ) : „ Hovmod er hadelig for Herren og Menneskene". — Den Hofferdige bliver skarp og uretferdig i sin Dom over Andre for felv at tunne indtage det overste Trin .

<h4>816</h4>Kors og Glede med hinanden . De boe i eet Huus , de stane i et inderligere og fortroligere Forhold til hinnnten end til noget andet Menneske . Og dog kommer det blandt utallige Par aldrig til nogen <b>sand</b> Enighed . Mange lere aldrig rigtig at forstaae hinanden . Vi ville slet itke tale om dem , som en told Beregning har fort fanunen , hvor egenlig itte Personene , men deres Midler og Eiendom ere blevne viede sammen . Men <b>selv</b> imellem dem , der af eget Valg og naturlig <b>Kjerlighed</b> have ratt hinanden Haanden , kommer det ofte aldrig tit nogen rigtig Forstaaelse . Hiin Begeistring i Forlovelsesdagene , da man altid svevede over Jorden , da man dromte og svermede , da man hellere vilde doe end opgive sin <b>Kjerlighed</b> , hvor snart tjolnes den ille hos mange Par ! Michal , Sauls Datter , havde elsket David . Den unge Helt var en Mand efter hendes <b>Hjerte</b> . Hun frygtete ille sin Faders Vrede , hun blev Davids <b>Hustru</b> . Da Saul efterstrebte ham , reddede hun hans Liv . Da Morderne trengte ind i Huset , havde hun allerede hjulpet David til at flygte . Fjenderne fandt istedetfor ham en af hendes Huusguder , som hun havde lagt i Sengen . Da David endelig var bleven Konge over Israel , og Sauls Tilhengere vilde slutte Fred med ham , var det hnns eneste Betingelse , nt man skulde give ham hans <b>Hustru</b> Michal tilbage , som Saul havde frcitaget ham i Vrede og giftet med en Anden . Og dog var der ingen rigtig Forstaaelse mellem de to ZEgtefolt . Den stolte Kongedatter vilde not have den nnge Helt til Mand , men itte den ydmyge Guds Tjener . Da David i sin Glede over Herren dandsede foran Pagtens Art , fpcttede den stolte <b>Kvinde</b> hnm . Efter hendes Mening hnvde hnn ved denne Leilighed beganet noget Uverdigt. Deres LEgtestnb vnr siden den Tid kun en told Overeenstomst . Og det Enmme sinder tesverre Sted i mange Tilfelde . Den forste Kjerligheds hurtig ! udsprungne Blomst visner , og ofte tan der siges om den : „ Dens Sted tjender man ikle mere . " — Derfor ere netop de forste Aar i 3 Egtestllbet llf san stor Betydning . Israel havde den Lov , at Enhver , der havde bygget sig et Huus eller tåget sig en <b>Hustru</b> , var fri for Krigstjeneste i det forste Aar derefter . Der laa den dybe Betydning heri , at man skulte have Tid til at leve sig ind i hinanden , at Charattererne

<h4>1230</h4>Fader- eller Modertjcerlighed , som Du viser dem , med barnlig Tatnemlighed . Nu kan vistnok Ingen rcekke Haanden paa samme Maade til Alle . Men der findes Huller i enhver Menighed , forn Kjcerligl , eden stal udsylde ; der findes Forceldrelose i enhver Menighed . Og hvem har vel et naturligere Kald til at vise <b>Kjcerlighed</b> mod saadanne forladte Born end den , hvem Gud ingen Aftom har stjcenkct ? I det ene Huus mangler Bornene , i det andet mangler Forceldrene ; og Intet er tydeligere , end at Gud vil lade den ene Mangel bode paa den anden . Men har en Familie med Trostab cmtaget sig forceldrelose Born , saa behover den sikkert itte at klage over Tomhed og Eensomhed i Alderdommens Dage ; thi der gives efter gammel Erfaring ingen taknemligere Mennester end Forceldrelofe , forn have gjenfundet en Fader og en <b>Moder</b> . Men ville I virkelig tåge et fremmed Barn til Eder , saa gjorer det , medens det endnu er meget ungt ; thi da bliver dets Opdragelse meget lettere for Eder , og det binder sig fastere til Eder . Halvvorne Adoptivborn blive ofte ved at vcere Fremmede i Hnfet trods al <b>Kjcerlighed</b> og Trostab . Vel opgive LEgtefolt , forn endnu ere i deres kraftige Alder , nodigt det Haab , at Gud vil velsigne dem med Born , og opscette derfor et saadant Stridt fra det ene Aar til det andet . Og Gud tan vistnok endnu opfylde deres Haab ; men han over iscer Barmhjertighet» mod dem , som <b>selv</b> have ovet den . Og hvad der maastee afkortes i de andre Boms Arvedeel for det fremmede Barns Skyld , det kan den rige Gud erstatte tusindfold . Jeg horte for itte lcenge siden en barnlos , gammel <b>Kone</b> beklage sig saaledes : „ leg og min salig Mand have forsomt Noget . Vi skulde allerede for 25 Aar siden have tåget ? t fattigt , forceldrelost Barn til os . Men da vi indsaae vor Feil , var det for sildigt . Vi vare for gamle til at opdrage et lille Barn , og havde vi tåget et storre til os , saa vilde den Smule Arv , vi efterlade , have faaet eu altfor stor Betydning for det ; vor Formue havde da maastee ligget det mere paa <b>Hjerte</b> end vi felv . Jeg foler godt , at Alle , forn i de senere Aar have villet stutte sig til os , havde Bihensigter ; men jeg foler ogfaa dybt , hvor eeufom jeg staaer i min ncermeste Kreds . " — Man kan her i <b>Sandhed</b> sige : „ Hvad Du gjor , det gjor snart ! " Og da lade

<h4>1262</h4>vor egen og vore Kjceres Frelfe . O , stjcenk os disse dyrebare Goder for din Varmhjertigheds Skyld . Amen . Kjoere Lcesere ! Fra Begyndelsen af har Gud henviist Mennestet til <b>Kjcerlighed</b> . Samfund og gjensidig Hjoelp . Denne Trang til <b>Kjcerlighed</b> — Mennestet maa elske og treenger atter til Andres <b>Kjcerlighed</b> — er <b>Moder</b> til Venstabet, der er ligesaa gammelt som Mennestesloegten . De gamle , hedenste Folk , iscer Groekerne og Romerne , holde Venstabet hoit i det bliver ofte besunget af dem , og Navnene paa trofaste Venner blive bevarede fra Slcegt til Sloegt . En af deres Vise talder en trofast Ven det bedste Stykke i Huset , og en anden kalder ham den bedste Lcege . Ogsaa hos Israels Folk soettes der stor Priis paa det trofaste Venstab . Forholdet mellem Jonathan og David , mellem Kongesonnen og Hyrden , er som en smuk , gron Plet midt i den sorgelige Srken , som frembyder sig for <b>Blikket</b> i Sauls sidste Dage . Davids Klagesang over den faldne Ven overgaaer alt Lignende i <b>Kjcerlighed</b> og Inderlighed . I hans Son Salomons Ordfprog omtales en Ven som den kosteligste Stat , saaledes som ogsaa vor Tert viser ; ja , det hedder endogsaa ( Ordsftr . IN , 24 ) : „ Ter er den Ven , som holder fastere end en Broder " . Og den gamle Cirach , der beskriver Livet , som det er , der forer Guds Viisdom ind i alle Livets Forhold , der udproeger Aabenbaringens Guld i Hverdagslivets Smaamynt , tan ikke noksom prife , hvilket stort Gode en trofast Ven er . Han skriver ( 6 , 15 ) : „ En tro Ven er en stoerk Beskyttelse ; hvo den haver sundet , han haver sundet et Liggendefce . En trofast Ven er ikke at betale med nogen anden Ting . En tro Ven er Livs Lcegedom, og de , som frygte Herren , stulle finde ham . " I det nye Testamente omtales Venstabet kun lidet . Pilatus og Herodes , som bleve Venner paa den Dag , da Herren korsfcestedes, og hvis Venstab blev indviet ved ustyldigt Blod , fortjene ikte- at kaldes faaledes ; deres Venstab har heller ingen Dybde hcwt ; thi en told Steen og et Stytte raaddent Trce tunne ikke holde sammen . Men <b>selv</b> blandt de forste Christne er der kun liden Tale om Venstabsforbindelser mellem Enkelte . Hvoraf kommer det ? — Vi maae soge Grunden hertil i det herlige , broderlige Menighedsliv . De Christne havde eet <b>Hjerte</b> og een Sjoel ( Apostl . Gj . 4 , 32 ) .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111600006'target=_>, 1856, Guds Man</a><h4>2196</h4>2. Som Ven han for ham <b>moder</b> Med Kys og Favnetag , Ei <b>Kjcerlighed</b> ham noder Og er ei <b>Hjertes</b> Sag . Sin Mester fer han solgte . Og var forbunden nu Af Satans Magt fremtraadte , Forraadte saa sin Gud . Z . Falsk ' Brodre her stal mindes Om alt sit Smigreri; Af een Synd let man bindes , Af Satan holdes i ; O , naar man troer sig sikker , Guds Ord ei agtrr paa , Da let i Satans Stritter Saa skjult Gen her kan gaae . 4. Skal Godt , som monne gjsres Mod Jesu Mindste her , Engang dog vist bclsnnes ' ) — Til Mindelse og er : Det Onde som udoves Mod Jesu mindste Wt , Vil engang det fortrydes Af ham , som gjorde det 5. Saa vil vi nu ei agte Udvortes Form og Takt , Men ham vi vil betragte . Som Doden <b>selv</b> har smagt : Oprigtig han sig viiste Mod os til sidste Stund ; Gid vi og hans Forloste Var <b>Sandhed</b> i vor Mund . ° ) 6. Hvor salig den mon ' verre , I hvis Aand ei er Sviig ; 4 ) Et Jesus Sind at berre Som Barn i Naadens Tid I Tro og <b>Sandhed</b> blive Med Gud og Alle saa , Som Ordet os beskrive , O , Jesu ! lad mig faae ! 7. Da jeg i dig mig glcrder Og fryder inderlig , Jeg din Retfcrrdig qvceder En Sang saa frydelig « ) Om al din Nsd og Smerte Og dine Såar saa rod . Og Blodet af dit <b>Hjerte</b> , ' Som randt udi din Dsd . 8. Lad , for du dig lod gribe Til Lidelser og Nsd , Jeg mig derved maa vide Befriet ved din Deo . Som da , at dine Fiender , De lagde Haand paa dig : Saa lceg din Jesu Hcrnder , Til Signelse paa mig !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2007062210023'target=_>, 1862, Nat og Morgen</a><h4>5150</h4>af de moralske Lovbud ; men han ssgte at berolige sin Samvittighet ) dermed , at han med megen Strenghet » holdt Isdedommets Hovedbud :  Hsr , Israel ! Her , ' ren , vor Gud , Herren er een ; " ' ) og dette sikrede ham paa det Fuldsilrndigste hans Andeel i den tilkommen ^ de Herlighed . Vi forelcrste ham derpaa det selgende Vers om den <b>Kjcrrlighed</b> , som Gud fordrer af os , og spurgte ham , om han tilfredssiillede denne Guds For , dring , da han , som han <b>selv</b> havde sagt , stedse tienkte alene paa , hvorledes han kunde forssrge <b>Kone</b> og Born . Dette var Middelpunctet af hans Digten og Tragten , men ikke , hvorledes han kunde virre den eneste sande Gud til Behag . Disse vare derfor hans Guder , fordi han i sit <b>Hjerte</b> sysselsatte sig mere med dem end med Gud , og altsaa elstede dem mere end den sande Gud . Han maatte derfor <b>selv</b> indsee , at han ikke tjente den Herre Jehova alene , men havde fiere Guder . Herpaa svarede han , at saa strengt tog han det ikke med Loven . Han gjorde , hvad ban kunde , og Mere forlanger Gud ikke . Han bad hver Dag de foreskrevne Bsnner , iagttog Isdedommets Scrtninger og Skikke , hvilke han , som Medlem af Pag : tens Folk , havde at holde , og Forsoningsdagen , som han ganske tilbragte med Bsn og Faste i Synagogen , staffede ham Tilgivelse for Alt , hvad han havde forssmt eller feilet . Nu viste vi ham af det gamle Testamente , hvorledes Gud fordrer ganffe Andet af en <b>sand</b> Israelit , og at han slet ikke byggede paa Mose og Propheternes Grund , hvilke altid drev paa Hjertets Omskjierelse . Hertil bemcerkede han , at dersom vi vilde omvende ham til Christendommen , vilde vor Umage virre forgjeves ; han var nu engang fsdt som Isde og vilde dse som Is ? de. Gud ihukommer eviudelig sin Pagt med Abraham . Han henlede derpaa det gamle Testamente , siog op Es . i 1,6 . : , 6. :  En Ulv stal holde sig til et Lam " : c . , og vilde af dette Sted overbevise os om , at ikke Jesus kunde virre den forjcettede Messias , da der endnu ikke er almindelig Fred paa Jorden . Efterat vi havde forklaret ham dette

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081903025'target=_>Nissen, Johan, 1863, J. Nissens Samtaler over Den bibelske Historie</a><h4>212</h4>blive rige . I Phil . 2 hedder det : Han forringede sig <b>selv</b> , egl . han ndtemte sig <b>selv</b> , d . e . afferte sig ganske og aldeles sin guddommelige Herlighed . Med Luther synge vi : Fattig lom Dn til Jorden ned , adlede Armod og Usselhed , fattig gjorde Dn Dig med Flid , rige blev vi til evig Tid . Men nåar et Menneste , som staaer heit over Andre , fornedrer sig <b>selv</b> af <b>sand</b> <b>Kjerlighed</b> til dem , saa prise og rose vi det , og Historien optegner det , foråt fjerne Tider tuuue vide det og prise det ; hvor meget sterre viser sig da ikke her Guds og Sennens Majestet i denne Fattigdom . V . 8 : Der var Hyrder , som holdt Nattevagt over deres Hjord . Det var om Natten , derfor hedder Julen ogsaa paa Tydst den indviede , hellige Nat ( Weihnacht ) . Det bliver ferst forkyndt for fattige Hyrder og ilke for de Fornemste eller Rigeste eller Lerdeste i Bethlehem . Hvorfor ikke ? Fordi Herren nu som altid seer paa Hjertet . Thi sittert vare disse Hyrder de bedste Mennesker i Bethlehem , som vi ogsaa senere stnlle see . Men de vare fattige og af ringe Stand . Herover tog Gud ligesaa lidt Anstot » som over Marias Armod . Det er det Hellige hos den store Gud , at ingen Person sanseelse gjelder hos ham , og det prise vi og glede os over . Vryd Dig ikke om , at Du klin er en simpel Pige eller en fattig Mand ; er dit <b>Hjerte</b> fromt og godt , faa er Du mere auseet hos Gud end den rigeste Herre paa Jorden . Saaledes aabenbarcr der sig altsaa netop i alle disse fattige Omstendigheder noget for os Herligt og Gledeliat . 2 ) Tet Store og Herlige i Jesu Fodsel viser sig ogfaa i den Deel , som hele Himlen tager deri . V . 10 : Frygter ikte ; thi see , jeg f o r k y n d e r E d e r en s t o r G l e d e , siger Engelen . Frygter ikke ; ak , vi maatte vel frygte , saalenge der ingen Forleser var ; hvorfor ? Nu stal Frygten ud af Hjertet , og Gleden drage ind . Som skal vederfares alt Folket ; Alle stnlle have Deel i denne Glede , Unge og Gamle , Fattige og Rige , Fornemme og Ringe , Lerde og Ulerde . Thi — det er Aarsagen til Gleden — Eder er idag en Frelser fedt , d . e . Jesns , der borttager Synden , som hindrer Saligheden . Som er den Herre Christns , d . e . den Salvede , der , Aab . 1,6 , ogsaa gjer os til Konger og Prester for Gnd . Herren , faaledcs kalder Engelen ham ; saaledes talde Disciplene ham , saaledes Sftaadommen ( David : Herren sagde til min Herre ) ; saaledes kalder hele Christenheden ham i Troesbetjendelsen , i den anden Artikel ; vor Herre Jesus , saaledes kalde vi ham her i Skolen , og saaledes enhver Christen , der veed sig i Samfnnd med ham . O , hvilke herlige Navne , der ret betegne det Store ved dette Barn og ved hans Gjerning . Gleder Eder derfor , Bern , og frygter ikke ! Op , gledes Alle , gledes nu , med Fryd opfylder Sind og Hv , i Herren Eder gleder ! ( Ps . Nr . 139 ) . V . 13. En himmelsk Herstares Mangfoldighet » lovede Gud . Det er atter en uforlignelig Herlighed , som vor Fortelling kundgjer os . Hele Himlen tager her Deel i Begyndelsen til Menn eskes leg tens Forlesning . Noget Saadcmt har aldrig sundet Sted , hverken fer eller senere , saalenge Verden har staaet ( Hvilket ? ) . Hvor stor maa da ikke Sennens Mennesteblivelse vere i Gnds og Englenes <b>Oine</b> ! Der er allerede Glede i Himlen hos Guds Eugle over een Synder ,

<h4>424</h4>et Ord , som Jesns har ndtalt om sig og sin Person . Hvilket ? — Ja , min Herre , min Konge og min Gnd ! jeg tilbeder Dig , thi Du er <b>Sandheden</b> , mine <b>Oine</b> see det , mit <b>Hjerte</b> feler det . 4 ) Og Hunlens Fugle komme og bygge Rede i dets Grene . Pf . Nr . 287 , 4 : Gnds Nige lignes ved et Fre — . Imellem Treets Grene sinder Fugleu Hvile om Natten , Beskyttelse for Uveiret og feler Glede over sin Nede . Det Samme paastaaes ogsaa om Guds Rige , Rom . 14 , 17 : Guds Rige er Retferdighed og Fred og Glede i den Helligaand. Vi ville cmfere Noget , forn iser er ioinefaldcnde . Naar en Fader i China itte kan . eller vil ernere sit nyfodte Barn , saa legger han det ud paa Gaden eller kaster det i Våndet . I Ostindien taste Medrene <b>selv</b> deres Bern i Grmgesstoden til 3 Ere for deres Afgnder . En afrikansk Konge selger aarligt mange af sine Undersaatter for Brendeviin og andre Ting . Naar en Mand doer i Ostindien , saa bliver hans Lig brendt , og hans <b>Kone</b> maa lade sig brende levende med. Jeg vil ogsaa cmfere et Udfagn af Koranen , Tyrkernes Bibel : Gjengjelder Forncermelser med Forncermelser ; kjemper saalenge med leder og Christne , til de bekvemme sig til at underkaste sig . — Men Christendommen lerer — — — .

<h4>432</h4>Paa samme Maade forholder det sig med Christendommen . Sverge vi , hvorledes er det menneskelige <b>Hjerte</b> beskaffent af Naturen , d . e . uden Christendom , saa staaer Svaret i Eph . 2 , 3 : Iblandt hvilke ogsaa vi Alle — . Altsaa af Naturen Vredens ( Guds Vredes ) Bern , gjere Kjedets , d . e . det syndige <b>Hjertes</b> Villie , vandre i vort Kjeds Begjerligheder . Hvorledes bliver nn det syndige <b>Hjerte</b> bestaffent , nåar Christendommens Snnrdeig tommer ind i det ? Det staaer i V . 5 : Gnd levendegjorde ogsaa os med Christus , der vi vare dede i Overtredelserne . Det menneskelige <b>Hjerte</b> forandrer da sin syndige Natnr og antager Christendommens guddommelige Natur . To Mend knnne gjere os dette tydeligere: Paulns og Johannes . Paulns gik ind i Husene i Jerusalem og tral baade Mend og <b>Kvinder</b> frem og overcmtvortede dem i Fengsel , Ap . G . 8 , 3. C . 9 , 1 : Han fnos med Trnsel og Mord mod Herrens Disciple . Hvilken Forbittrelse i det syndige <b>Hjerte</b> ! Da rerte Christendommen hans <b>Hjerte</b> og gjorde ham til et ganske andet Menneste . 1 Cor . 4 , 12. 13 : Overstjeldede velsigne vi ; forfnlgte taale vi ; bespottede formane vi . 1 Cor . 13 , 4. 5 : <b>Kjcerligheden</b> er langmodig , er velvillig , forbittres ikke , berer ikte Nag . Hvorledes forstaae vi dette ? Panlus svarer os <b>selv</b> i Phil . 4 , 13 : Jeg formaaer Alt i Christo , som gjer mig sterk . — Johannes . Vi lese Lnc . 9 , 52 — 56. Det var Stolthet » og Hevngjerrighet » ; saaledes berer det syndige <b>Hjerte</b> sig ad . Men hvor ganske cmderledes senere ! 1 Joh . 4 , 11 : I Elskelige , har Gnd saaledes elsket os ( hvorledes ? ) , da ere vi og styldige at elste hverandre . Altsaa , det er den ferste Virkning af Chriftendommcn i Menneskets In drc , at den forandrer den syndige Natnr , det syndige A jer te til Lighed med Guds Vesen . Iser er det Stolthet » og Hevngjerrig he d , hvori det syndige <b>Hjerte</b> fornemmelig viser sig , der ved Christendommen og kun ved Christendommen forvandles til Id myg hev og Sagtmodighed , saaledes som vi saae den hos Panlus og Johannes . Og endnu daglig see vi de samme Virkninger . Saaledes hos Bern og Voxne , nåar deres Hjerter rores af Christendommen ; den legnagtige Natnr forandres til guddommelig Natur , til <b>Saudhed</b> . De hevue sig itte , men tilgive , ligesom Gnd . De ere ikke stolte , men deres Hjerter blive , ligesom Guds Sens <b>Hjerte</b> , sagtmodigc o . s . v. Det nye Test . har mange forskjellige Udtryk for denne megtige Virkning af Christendom men ( Hvilken ? ) . G jen f edel se i fjerde Hovedstykte af den lille Catechismns . Den gamle Adam deer , og et nyt Menneske opstaaer i os . Den gamle Adam , Kjedet , det syndige <b>Hjerte</b> deer ; den , der deer , kan ikke mere udfere Noget , Synden har ingen Magt mere . Et nyt Menneste opstaaer , handler og virker ; altsaa det Modsatte . Christus lever i mig , siger Panlns , Gal . 2 , 20 , d . e . jeg har hans ( guddommelige ) Natur , hans Maade at tenke , at tale , at handle paa . Men have vi saaledes annammet Christi Natnr , saa blive vi derved deelagtige i den guddommelige Natur . Vi lese 2 Pet . 1 , 3 — B . Har Du ogsaa det , som Apostelen i V . 5. 6 regner til denne guddommelige Natur ? nu saa gjer , hvad han forlanger , „ anvend al Flid . " Endnn et Udtryk : et Guds Barn , Rom . 8 , 14. 15. Til alt dette vil , som

<h4>631</h4>Turde hau foragte Audre ? 1 ) Gud foragter Ingeu . 2 ) Hnn kjen dte ikke sig felv . — Nu roser hau , hvad hau gjsr . Jeg faster — . Det var det Lettere istedeufor det Svcerere : Dom , Barmhjertighet ? , Tro . Saaledes tale Mange endnn . Jeg har ikke stjaalet fra Nogen ; jeg gaacr flittigt i Kirke , giver til de Fattige , lceser gjerne i Bibelen . Godt ; men hvorledes staaer det til med dit <b>Hjerte</b> ? Idmyghed , Tro , <b>Kjcerlighed</b> . Ps . Nr . 336 , V . 4 : Lad mig i Bogstavens Vcesen ei blive , som knn gjsr ndvortes cerbar og fun — . Hvorfra stammede nu Pharisceereus Bsn ? Fra St ol th ed ; han nedsatte Andre og ophsiede sig <b>selv</b> . Toldercns Bon . Gud , vcer mig Synder naadig ! Han kalder sig en Synder , d . e . En , der har handlet mod Guds Villie . Der er vel ogsaa noget Godt ved denne Tolder , men derpaa tcenker han slet ikke ; han har kun sine Synder for <b>Oie</b> og danner derved en fuldstcendig Modscetning til Pharisceeren . Hvorledes ? Altsaa Tolderen erkjender sin Synd . Snnledes synge vi i Psnlmen Nr 331 : O prsv Dig , <b>Hjerte</b> , prsv Dig vel , er blind Du for din Svndegjceld , dit Ansvar Du forsger . — Hau stod langt borte ( fra det scedvcmlige Bedested ) og vilde ikke oplsfte Oinene til Himlen , men slog sig paa sit Bryst . Hvorfor ? Jeg har krcenket Gud , der har gjort mig faa uendelig meget Godt ; jeg tsr ikke ncerme mig til ham ; jeg har fortjent Straf . Stam og Anger . — Han snster sin Skyld ophcevet ; men han fsler , at hans egen Fortjeneste ikke kan udstette den , at han Intet har at stole paa . Vcer mig naadig . Han grunder sit Haab paa Guds Naade . Denne korte Bsn indeholder altsaa tre Ting : 1 ) Erkjendelsen og Bekjendelsen af Synden . 2 ) Skam og Anger . 3 ) Tro paa Guds Forbarmelse . Et Menneske , der beder , staaer for Guds Aasyu , taler med Gud og bsr derfor aldrig gjsre meget Vcesen af hvad han er , eller hvad han har . Hvorfor ikke ? Det er Admyghed . Saaledes var Tolderen . Denne Idmyghed , der er det Modsatte af Pharisceerens Stolthed , findes hos alle store Mcend i den hellige Skrift . Jeg stal sige deres Ord , ncevn mig saa deres Navne . Jeg har begyndt at tale til Herren , alligevel jeg er Stsv og Aste . — Jeg er ringere end alle de Mistundheder og al den Trofasthet » — . Hvo er jeg , at jeg skulde gaae til Pharao — . Hvo er jeg , Herre , Herre , og hvad er mit Huus , at Du har fsrt mig hidindtil . — Jeg er en liden Dreng , og jeg veed ikke at gaae ud eller gaae ind ; saa giv din Tjener et forstandig : <b>Hjerte</b> til at dsmme dit Folk og med Forstand nt stille imellem Godt og Ondt . — See , jeg er Herrens <b>Tjenerinde</b> — . Desnden anbefales og roses Idmygheden i flere stjsnne Sprog i den bellige Skrift . Hvo sig <b>selv</b> ophoier — . I Micha 6 , 8 staaer der udtrykkeligt , at Du stal ydmyge Dig og vandre med din Gud . Christus begyuder siv Bjergprcediken med at prise de Fattige i Acmden salige , thi Himmeriges Rige er deres . Derfor ydmyger Eder under Guds vceldige Hacmd , at han maa i sin Tid ophsie Eder . 3 ) Virkningen af deres Bon . Havde en Isde vceret tilstede , da Pharisceeren og Tolderen gik op til Templet , og man havde spnrgt ham , hvem disse To vare , havde han maastee svaret : den Ene er et fromt Menneske , den Anden en Ugudelig ; thi Menneskene see Pan det , der falder i Oinene , paa det Idre . Men spsrge vi nu , hvad Gud tcenker

<h4>694</h4>og Glceder og ikke kjender til noget Vedre . O , hvor fattig og elendig er et saadcmt Menneske i den anden Verden ! Tcenk paa den rige Mand og Lazarns . Men hvad er bedre end denne Verdens Gods ? Rig i Gud , V . 21. Allerede Salomon siger , Ordspr . 13. 7 : Der er den , sem holder sig rig og har slet Intet ; og den , som holder sig fattig cg har meget Gods . Ligesom der nemlig gives jordiske Goder , saaledes gives der ogsaa- himmelske . Nn knnne I let <b>selv</b> forklare dette Ordsprog . Men Gnd giver os jordiske Goder og Rigdom , foråt vi derved stulle erhverve os himmelske Goder eller blive rige i Gud . Hvorledes stulle vi gjsre dette ? Lader os betragte deu rige Boude . Haus Marker havde baarct godt , og derfor stnlde han have lovet og takket Gnd . Saa havde hans Sind og hans Tanker vendt sig til Gud , og da havde hau ilke alene glcedet sig over Gaverne , men ogsaa over Giveren af alle gode Gaver . Saa havde han samlet sig en Rigdom i Gud , der ogsaa havde vedvaret efter Dsdeu , nåar vi evigt stnlle love og prise Gnd . — Han stnlde fremdeles have tcenkt : Den kjcere Gud har velsignet mig saa rigeligt , jeg vil samle mig Skatte i Himlen dermed og gjsre Godt mod Andre , saaledes som han har befalet . Es . 58 , 7 : Bryt » dit Brsd til den Hungrige , briug fattig Flygtuiug i Huus ; seer Du eu Nsgen , Du klcede ham , og drag Dig ei tilbage fra din Nceste . Den , der er rig i Gnd , han vandrer med Gnd og for Gud , han sver alle sine Gjerninger som for Guds <b>Oine</b> , efter Gnds Bnd . Det bimmelste Sind er ilke bekymret , V . 22 ; er stedse vaagent og beredt , V . 35 - 37. Et saadant Sind see vi bedst hos Jesns <b>selv</b> . Al hans Tale er opfyldt af Tanken om Gud , f . Ex . forn Barn : Vidste I itte , at mig bsr at vcere — . Her : Ingens Liv bestaaer i — . Da han sad til Bords : Eet er fornsdcnt ; byd Fattige , Krsblinger , Halte , Blinde , saa stal Du vorde salig . Da han gik van Mnrken : Seer til Himlcns Fugle — . Dn hau gjorde vel mod Andre : Min Fader arbeider indtil nn — . Da han led : Skee ikke min Villie — . Saaledes ogsaa hans Apostle , Phil . 3 , 20 : Vort Vorgerstab er i Himlene . Hvad mener Apostelen ? Saaledes ogsaa Dn : Vend tidligt dit Sind hen til dit sande Hjem . „ Vender <b>Hjerte</b> , Sind og Hv fra den tomme Verden bort . " Hvorledes tan Dn vise et himmelsk Sind i Skolen ? Hvorledes er Dn rig i Gnd , nåar Dn spiser og drikker ? 1 Cor . 10 , 31 : Hvad heller I derfor cedc eller drikke , eller hvad I gjsre , ra gjsrcr Alt til Guds Wre . Col . 3 , 17 : Alt , hvad I gjsre i Ord eller i Handling , det gjsrer Alt i den Herres lesn Navn , takkende Gnd og Faderen formedelst ham . Hvorledes stulle en Pige og en <b>Kone</b> vcere rige i Gnd ? Lceser 1 Pet . 3 , I — 3. Hvorledes stulle Tjenestefolk vcere det ? Eph . 6 , 5 - 8. Hvorledes Herrer ? Eph . 6 , 9. Af Gal . 5 , 22 kjeude I de vi himmelske Goder : <b>Kjcerlighed</b> , Glcedc , Fred , Langmodighed , Mildhed , Godhed , Trofasthcd , Sagtmodighed , Afholdenhed . Den , der har dem , cr rig i Gud og lider aldrig uogen Mangel her , saaledes som Apostlene havde erfaret det : Som bcdrsvede , dog altid glade , 2 Cor . 6 , 10 ; vor Trost er ovcrflsdig ved Christum , 2 Cor . 1 , 5 ; opfyldte med nl Guds Fylde , Eph . 3 , 19. Hertil kommer Hnnbet om en nfortrcenkelig og übesmittelig og nforvisnelig Arv , som er bevaret i Himlene , 1 Pet . 1 , 4. — Han lider heller ingen Mangel hisset , om ogsaa

<h4>899</h4>Te Skriftklogcs Indvcnding . Da de Skrifttloge hsrte dette , sagde de : Hvo er denne , som taler Gudsbespottelse ? Hvo kan forlade Synden uden Gnd alene ? Vistnok kan Gnd alene forlade Synden og intet Menneske . Men allerede i det gl. Test . var det sagt , at Verdens Frelser skulde borttage Synden . Saaledes hedder det i Dan . 9 , 24 : Der er halvfjerdsindstyve Uger bestemte over dit Folk og over din hellige Stad til at forhindre Overtredelsen og til at besegle Synder og til at forsone Misgjerninger og til at lade komme en evig Netferdighed . Hvis nn disse Skrifttloge havde erkjendt Jesus fer nt vere Verdens Frelser , san havde de cgsan indrsmmet , nt hnn havde Magt til at forlade Synder . Vien dette ( hvilket ? ) vilde de ikke , og heri viser sig deres onde <b>Hjerte</b> . Ogsaa for Joseph var det forkyndt , at Jesns stnlde frelse fra Synden , Matth . 1 , 21 : Du stnl kalde haus Navn Jesus , thi han stal frelse sit Folk fra deres Synder . Og Johannes den Dsber havde sagt : See , det Guds Lam — . Jesns rctflerdiggjor sig . Matth . 9 , 4 : Jesus såne deres Tanker . Det Indre , som er skjult for menneskelige <b>Oine</b> , er ikke skjult for Hjertetjeudcreu. Hvi tente I san ondt i Eders Hjerter ? Deres Tanter ( hvilke ? ) ere allfaa onde , vi have nylig scet hvorfor . De ville ikke anerkjende Jesns for den , han er , og den , han viste sig at vere . „ En Kraft gik nd fra ham " ; de see denne Kraft , men ville dog ikke troe . De see ogsaa her hans forbarmende <b>Kjcerlighed</b> mod Synderen , men det rsrer dem ikke . Ogsaa os lyser lesn guddommelige Kraft eg <b>Kjerlighed</b> imsde fra ethvert Blad i det nye Test . , eg hvormcmge er der dog ilke , forn ei ville erkjende Jesns for deres Frelser ! — Da Jesus nn har opladt dem deres onde <b>Hjerte</b> , vil han i sin <b>Kjcerlighed</b> ogsaa ssge dem og forlade dem deres Synd og vere deres Frelser . Derfor siger han nn videre : Hvilket er lettere nt sige : dine Synder ere Dig forladte — ? Begge Dele ere lige svere , begge ere kun mulige for Gud . Meu unar Jesus tan gjsre del Ene , saa kan han ogsaa gjore det Andet . Og nn ndfsrer han Underet paa den Vertbrndne : Staa op , tag din Seng og gak til dit HnusC Og han stod op og gik bort til sit Hnns . Hnn , hvem nylig Fire hnvde bnaret , gaacr nu paa lefu Bud og berer <b>selv</b> deu Scug , der nylig havde baaret ham . — Hvortil dette Under ? Jesns siger det <b>selv</b> : At I stulle vide , at Menuesteus Ssn har Magt paa Jorden at forlade Synder — . Den legemlige Helbredelse stal vise , at han i <b>Sandhed</b> tan helbrede nnndeligt , d . e . forlade Synder . Herren begynder her med den aandelige Helbredelse som det Allervigtigste ; thi Mennesket kan nok blive saligt uden Suudhed , Rigdom og LEre , men ikke uden Syndernes Fcrladelse . Leg Merke til , at Jesus siger : Menuesteus Ssn har Magt paa Jorden at forlade Synder ; thi det er en stor Trsst for os at vide , at der er kommet En ned fra Himlen , som har Magt til at forlade Synder her paa Jorden ; at Herren har overdraget denne Magt til sine Disciple , Joh . 20 , 23 : Hvem I forlade Synderne — ; og at der predikes Syndernes Forladelse i den christne Menighed . Luther siger : Det er saudelig eu stor Magt , som en Christen har , at han kan sige til en Anden ( som Presten gjsr ved Striftemaalct ) : Ver frimodig , tro kun , hvad jeg siger Dig i Jesu Navu : dine Synder ere Dig forladte . Og i den lille

<h4>1112</h4>giver Jesus sig tilkjende for hende , hvad han dengang endnu itte havde gjort for nogen Isde , og <b>Kvinden</b> troer hans Ord med stor Glede . Hvoraf knnne vi see det ? Deraf , at hnn lader sin Vandkrutte staae og gaacr til Staden og siger til Folkene , der : Kommer , seer — . Jeg har Mud at wdc , den I itte kjende . Min Mad er , at jeg gjsr hans Villie , som mig ud send te , og fuldkommer hans Gjerning . Disse Ord vise den Iver og Glede , hvormed Jesns ssger at gjsre os salige . Det er min Mad ; deri ligger , at det er Jesns en Glede , en Vederkvegelse , en Fornsdenhet » at gjsre Guds Villie . Men Gnds Villie er netop , at Ingen stal forlades , men at Alle stulle komme til Omvendelse , og denne Villie vilde Jesus til euhver Tid opfylde . Hvilken Troskab og <b>Kjcerlighed</b> ! Og saaledes er Jesus euduu : Div Hyrdestemme , o hvor mild den kalder hver , som farer vild . Hvorledes har Jesns taldet og ssgt Dig ? Ved Dciaben , ved Guds Ord . Du foer vild . d . e . Du ssgte Frelse og Glede og Salighed i Syuden : i Hcevngjerrighed , Lsgn , Stolthet » o . s . v. Jeg har Mad at ede , den I ikte kjende ; thi den Salighed , at omvende en Synder , kjende knn Faa . Mon jeg vere min Broders Vogter , sige Mange . Hvad betyde disse Ord her ? Da Judas vilde ssge Trsst hos Ippersiepresterne , sagde de til hnm : Hvad kommer det os ved ? see Dn dertil . Saaledes ville vi itte tale . Du kaldes en Christen og stal efterfslge Christus ; hvcrledes og hvornaar gjsr Du det da i det Punkt , hvorom vi her tale ? Og endnn et Skriftsted herom , som jeg ret vil legge Eder paa <b>Hjerte</b> , Jak . 5 , 19. 29 : Brsdre , dersom Nogen iblandt Eder er faren vild fm <b>Sandheden</b> , og Nogen omvender ham , den vide , at hvo som omvender en Synder fra hans Veis Vildfarelse, han frelfer en Sjel fra Dsden og stjnler Synders Mangfoldighet». Vi have <b>selv</b> hort og vide , at denne er sandelig den Vcrdcns Frelser , den Christus . Nn havde de <b>selv</b> hsrt det og vidste det , fsr tjendte , de det tuu af <b>Kvindens</b> Fortelling . I hsre ogfaa fortelle i Skolen om Jesus , og om at han er Vcrdcns Frelser ; men I stnlde <b>selv</b> hsre det og <b>selv</b> vide og erkjende det . Hvorledes steer det ? Naar I lade Eder sige af Jesus , at I ikke ere saaledes , som I skulde vere for Gud , hverken i Tanker , Ord eller Gjerninger . Naar I late Eder sige , nt der endnu mangler Meget i , at Christns boer i Eder , d . c . at Eders Tanker , Ord og Gjerninger nfspeile Jesu hellige Liv . Hvor findes haus Sagttnodighet » , Idmyghed , Tjenstagtighcd og Fromhet hos Eder ? Naar I fslc dette , kj . 8. , sna crtjcnder fremdeles , at Christus kan give Eder Alt , hvad der fattes Eder . Han har elsket Eder indtil Doden og har ladet sit Liv for Eders Synder ; lad da denne <b>Kjcerlighed</b> vette Eders Gjenkjerlighed , saa I bestilte Eder paa alt Godt ; med Panli Ord : Christi <b>Kjcerlighed</b> tvinger os . Og dertil stal det ogsaa komme med Eder ^ paa det at , forn Panlns figer i 1 Cor . 2 , 5 , Eders Tro ikke stal vere i Mennestens Viisdom , men i Guds Kraft . Hvad mener Apostelen ? Kort sagt : Christus soger Eder , saa lader Eder da siude af ham . Kommer da , vi vil os samle om det Livsens Kildeveld, tommer Unge , kommer Gamle , kommer hver en msdig Sjel !

<h4>1128</h4>der have begaaet stemme Forbrytelser ; men det bliver itte <b>sandt</b> , nåar vi tåge Hensyn til mange Forbrydercs nnverende Liv . Troe I maastee itte , Bsrn , at der gives Tugthnusfanger , som ere frommere end vi ? Hvori overgaae de os maastee ? I Bodferdighed , Tro og <b>Kjcerlighed</b> . Ja , det er virkelig Tilfeltet , og jeg knnde fortelle Eder Exempler derpaa . Lader os derfor see til , Bsrn , at det ikke stal opfyldes paa os , hvad Jesns sagde , Matth . 21 , 31 : Toldcre og Skjsger gaae for Eder i Guds Rige . Hvornaar nemlig ? Det kan ikke siges for ofte : Naar de vife Bodferdighed , Tro og <b>Kjcerlighed</b> , og Du itle gjsr det . Derfor stal Du hos Dig <b>selv</b> og Andre legge Merke til disse treude Ting og glede Dig , nåar Du sinder dem hos en forhenvercnde Synder ; thi der er Glede i Himlen over een Synder , som omvender sig . 2 ) Jesus . Han opfylder her Hensigten med sin Sendelse til Jorden : Mennestens Son er kommen at ssge og frelse det Fortabte . Han er her den . der ndstetter og tilgiver Synden ; og med hvilken Venlighed og <b>Kjcerlighed</b> skjenker han itte den dybt bekymrede Sjel Naade , Ro og Trsst ! I V . 47 siger han hsitideligt : Jeg siger Dig , hendes mange Synder ere hende forladte . Derpaa udtaler han det trsstende Ord til hende <b>selv</b> i V . 48 : Dine Synder ere forladte ; og foråt der itte stnlde opstaae Tvivl hos <b>Kvinden</b> ved de Tilstedeverendes Tanker og Tale , som itle vilde indromme Jesns Net og Magt til at forlade Synder , V . 49 , siger han endnn engang , V . 50 : Din Tro har frelst Dig , gak bort med Fred ; dit <b>Hjerte</b> stal nn itte mere engstes over din store Skyld , men i den tilsagte Syndsforladede stal din Sjel finde Ro og Fred med Gnd . Thi det gaacr , som det hedder i Rom . 5 , 1 : Retferdiggjorte ved Troen ( ved Troen paa , at Jesus kunde , tilgive hendes Synder og havde gjort det , var <b>Kvinden</b> bleven retfcerdig for Ond og Jesns ) have vi Fred med Gud ( thi vort <b>Hjerte</b> fordommer os nu ikke mere ) ved vor Herre lesnm Christnm . — Denne Begivenhet » er optegnet , foråt vi skulle holde os til Jesus , ncmr vi engstes over vore Synder og trenge til Trsst og Syndernes Forladelse; Ps . Nr . 434 , V . 3 : Om i en msrk og bange Stund saa mange Synders Minde gaacr frem af Sindets dunkle Grnnd , min Sjel dog Fred stal finde ; jeg veed vel , at min Synd er stor , men stsrre er det Naades-Ord : den Dig forladt stal vere ! — Saa ville vi da ogsaa ligesom denne Synderinde trsstes ved Jesu Venlighed og Huldsalighed ; thi eudnu bestaudig lyde Jesu Ord til os : Dine Synder ere forladte . Hvor , i hvilken hellig Handling bydes denne Trsst Eder ? I Skriftemaalet , i Nadveren . Livsalig er den Tale om Sjelehyrden god , som os fra Dsdcns Dale forlsste med sit Blod , som end fra Hunlens Borg sit Naade-Aar forkynder for hver vildfaren Synder og leger al hans Sorg , Ps . Nr . 407. 3 ) Synderinden . At <b>Kvinden</b> var en Synderinde , var Noget , som Alle i Vyen vidste ; hnn havde formodentlig fort et acibenbart forargeligt Liv , maastee bedrevet Utugt og Hoer . Men nu viser der sig Noget hos denne <b>Kvinde</b> , som gjsr hende velbehagelig for Gud og Jesus , og det er , som vi allerede have antydet , hendes Bodferdighed og Tro og tatnemlige <b>Kjcerlighed</b> . — V . 38 : Hnn veder Jesu Fsdder med Taarer , med Bodstacirer , som vi kalde dem . Vi see her , hvor

<h4>1329</h4>ganske andcrledes . <b>Kvinden</b> , som var greben i Hoer , Joh . 8 , fordsmmer han ikke , thi , Lue . 9 , 56 , Menucstens Ssn er itte kommen at forderve Menneskers Sjele , men at frelse dem ; itte at dsmme Verden , men at Verden stal blive salig ved ham . Han siger : Den , som er syndelss iblandt Eder , taste fsrst Stenen pan hende ; hnn fordsmmer hende ikke , men tilfsier snn ogsaa : Gak bort og synd ikte mere . Den samme Adferd viser han mod den Syge ved Vethesda Dam , Joh . 5 , 14 ; fsrst hjelper han ham og derpaa formaner han : Synd ikke mere , at Dig ikke noget Verre stal vederfares . Som Frngt af denne <b>Kjcerlighed</b> til Gud og Meunestene fremtrede ifer tyende Egenskaber hos Jesus , med Hensyn til hvilke han <b>selv</b> fremstiller sig forn et Forbillede for os , nemlig Sagtmodighed og Idmyghed . Lerer af mig , thi jeg er fagtmodig og Ydmyg af Hjertet . Jesus har aldrig hevnet sig . Han satte engang Disciplene irette , fordi de vilde lade Ild falde ned fra Himlen over en samaritans ! By , og sagde , Lue . 9 , 55 : I vide itte , af hvad Aand I ere ; thi Mennestens Ssn er itte kommen at forderve Menneskers Sjele , men at frelse dem . Han stjendte itte igjen , da han blev overstjendt, men overlod Hevnen til Gud , ja <b>selv</b> paa Korset bad han efter Spaadommens Ord i Ef . 53 for Overtrederne . — Jesns har fremdeles aldrig gjort Noget afTrods eller Lyst til forfengelig 3 Ere . Han fsgle itte LEre af Mennesker , men af Gud ; og i Tro paa Gud og af <b>Kjcerlighed</b> til Menneskene fornedrede han sig <b>selv</b> og ventede kun sin Ophoielse af Gud alene . I at vife Sagtmodighed og Idmyghed har hall fornemmelig fremstillet sig <b>selv</b> som et Msnster og Forbillede , fordi han vidste , hvor vanskelige disse tyende Dyder ere for det natnrlige Menneskehjerte , og at de knn blive mulige at udove i Kraft af hans Forlssning , nåar Mennesket treder i Samfund med ham . — <b>Sandheden</b> gik ham over Alt . Hnn pnnviser i Joh . 8 , 44 , at Djevelen er Lsgnens Fader . Han siger ikke : Jeg er scmddru , men : Jeg er <b>Sandheden</b> , t > . e . <b>Sandheden</b> er ham subjektiv , er hans Vesen , er Eet med hans Vesen . Men <b>Sandheden</b> er ham ikke blot den rigtige Tenkning og Erkjendelse , men Overensstemmelse mellem hans og Guds Villie ; den fuldtomne Overensstemmelse mellem <b>Hjerte</b> , Ord og Gjerning indbyrdes og med Gnd . Hvad han erkjender for at vere gnddommeligt , det siger og vil og tenker og gjsr han ; og aldrig „ blev der funden Svig i hans Mund " , 1 Pet . 2 , 22. I Omgang med Andre har Herren ingen seregne Regler fnlgt . Han har gledet sig med de Glade , som ved Bryllupet i Ccma , og ssrget med de Ssrgende . Hans <b>Hjerte</b> har veret aabent for Venstab , og han kalder <b>selv</b> Lazarns for sin Ven : Lazarns , vor Ven , er sovet ind . Hans Samtaler med Kvinter ntnierte sig ved stor Fiinhet og Inderlighet » ; man tenke kun paa hans Samtale med den samaritanske <b>Kvinde</b> og hans Forhold til Lazarns ' s Ssstre . Der findes heller ikke en eneste <b>Kvinde</b> i de evangeliske Fortellinger , der var fjcndtlig sindet imod ham , ja <b>selv</b> Jerusalems Dsttre gret » over ham . Det var eu <b>Kvinde</b> af Folket , der oplsftede sin Rsst og sagde : Salig er det Liv som bar Dig , og de Bryster , som Dn diede . Det var Kvinter , der fulgte med ham og ssrgede for hans Fornsdenheder , Lnc . 8 , 1 - 3 ;

<h4>1772</h4>hvilken Inderlighet , elskede han ikte Herren — nåar han tun kunde have den Glede , at Isderne bleve frelste . Hvilken <b>Kjerlighed</b> ! — 2 Cor . 2 , 4 : I megen Kummer og med beklemt <b>Hjerte</b> strev jeg Eder til under mange Taarer , ikke at I skulde blive bedrsvede , men paa det at I stulde kjende den <b>Kjerlighed</b> , som jeg overstsdigen har til Eder . — 1 Cor . 4 , 14. 16 : Jeg paaminder Eder , som mine elskelige Bsrn ; jeg avlede Eder i Christo Jesu ved Evangelium . — 1 Thess . 2 , 7. 8 : Vi omgittes mildt med Eder ; som en Amme pleier sine Born , saaledes ere vi af inderlig <b>Kjerlighed</b> til Eder villige til at dele med Eder ikke alene Guds Evangelium , men ogsaa vort eget Liv , fordi I ere blevne os elskelige . — Altsaa ligesaa omhyggelig , betenksom , om , kjerlig og mild som en <b>Moder</b> , der giver sit Barn <b>Die</b> ; tan der vel tentes noget smnttere Billede ? — Til de LEldste i Menigheden i Ephesns siger han , Ap . Gj . 20 , 31 : Kommer ihn , at jeg har ' itte afladt i tre Aar , Nat og Dag , at pacuninde enhver af Eter med Taarer . 1 Thess . 3 , 10 : Vi bede indstendigen , Nat og Dag , at vi maatte see Eders Ansigt og raade Bod paa Eders Troes Mangler . Og med hvilken Alvor og <b>Kjerlighed</b> anbefaler han ikke Menighedenes Vel til de i Milet forsamlede LEldste , Ap . Gj . 20 , 28 : Saa giver Agt paa Eder <b>selv</b> og den ganske Hjord , iblandt hvilken den Helligaand satte Eder som Tilsynsmend, at vogte Guds Menighed , hvilken han forhvervede med sit eget Blod . * ) — Med den samme <b>Kjerlighed</b> , som han viste mod hele

<h4>1841</h4>hvorfor ? Fordi <b>Kjcerligheden</b> gjsr den let , og den Korsfestede er min <b>Kjerlighed</b> * ) . Born , I ere dyrekjobte , vorder ikle Menneskers Trelle . Christi Kors siger til Eder : Lever i Verden , men ikke med Verden ; hvad vil det sige ? Gid Langfredag maa minde Eder om og opmuntre Eder til at doe med Chriftus , derved at I fornegte Verden , afdse fra den , og at den bliver korsfestetfor Eder . Ps . Nr . 488 : Bort , Verden , af mit Sind og <b>Oie</b> , din Lyst Du <b>selv</b> beholde maa , den tan ei mere mig fornsie , end Stovet , som jeg triner paa ! Hver elste , hvad hans Hu staaer til , jeg tun min lesum elste vil . 2 ) Gal . 5 , 24 : Men de , som hore Christum til , have korsfestet Kjodet ned Lysterne og Begjeringerne . Hvilke hore da Christnm til ? Hvorved blive vi hans Eiendom ? Hvad stal nu Doden , som han har lidt for os , bevirke efter dette Skriftsted ? Naar vi altsaa korsfeste Kjodet , som netop er de syndige Lyster og Begjerligheder , saa doe vi med Christus . Den Korsfestede var bestemmet og forbandet og afstyet , og netop dette stnlle ogsaa de syndige Lyster og Begjerligheder blive for os . Hvorledes kan nu Langfredag med dens levende Erindring om Jesu Dsd bidrage dertil ? De oude Lyster og Begjerligheder ( nevu uogle ) have netop bragt ham paa Korset , og Du kan endnu elske dem ? Du siger , at Du vil vere din Frelser tatnemlig for al hans <b>Kjerlighed</b> . Er det da Taknemlighed , siger Christus fra Korset til Dig , nåar dit <b>Hjerte</b> er opfyldt af Hevngjerrighet » ? Naar Du foragter din Fader og din <b>Moder</b> ? Naar Du i dine Lidelser ikke vil bsie Dig for Guds Villie , men vil have dine egne Onster opfyldte ? Lerer af mig . Hvilket Exempel viser Jesns os da i disse Tilfelte ? Saa lad da <b>Kjerligheden</b> til Jesns drage os sterkere end Synden . Hvad mener jeg ? O Herre , til jeg hist din LEre stal see i salig Himmelfred, lad i min Sjel det Billed vere , som vidner om din Kjerlighed, at hvor jeg hviler , hvor jeg gaaer , dit Kors , din Dsd i Hjertet staaer ! ( Ps . Nr . 200 , V . 5. ) Lader os bede Herren om et <b>Hjerte</b> , der tilhsrer ham alene og mere og mere afdoer fra Synden , ja : Lad mig dse , imens jeg lever , dse fra Kjsdets Lyster bort , da fra Graven msrk og sort jeg med ham til Himlen svever ; Jesu , jeg vil dse med Dig , at Du leve kan i mig ! ( Ps . Nr . 492 , V . 3. )

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008102303008'target=_>Harboe, Ludvig, 1844, Psalmebog, eller En Samling af gamle og nye Psalmer, til Guds Ære og hans Menigheds Opbyggelse, tilligemed Collecter, Epistler og Evangelier, Lidelseshistorien, Kirke samt andre Bønner</a><h4>2526</h4>Mel . Alenesle Gud i Himmerig : c . 342. Ophsiet Jesu ! sode Lyst For Sjcrlens klare <b>Die</b> , Jeg som mig altid ved dit Bryst Saa liflig kan foruoie , Jeg knatte for din Throne ned , Dg vil " din store <b>Kjcerlighed</b> Ved Frydesang ophoie . . 2. Du er et Lys , bestraale mig Mcd tdel Fryd og Glcede , Mit <b>Hjerte</b> derved skynder sig Paa Livsens Vei at trede Tag bort mit <b>Hjerte</b> , Sind og Sands ' Og fylt » ten med dit Lys og Glands Fra Hunlens lyse Scrde . 3. Du er miv sttkre Himmel-Vci Til Livsens Land at vinde , Hvo det forstaae han vandre ei Med Vcrdcns Flok t Blinde ; Ak ! lad mig ei indbilde mig , At kunne Himlen uden dig . Min sede Frelse ! sinde ! 4. Du , du er <b>Sandhed</b> <b>selv</b> , hvorpaa > seg fast og stadig bygger , I dig er Liv

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011072920003'target=_>Nathusius, Marie, 1880, Elisabeth</a><h4>102</h4>stod et Aieblik stille med Hjertet opfyldt af Tanken paa sin Lykke og paa sin nye Stands Pligter og Vcerdighed . ~ la , du stal verre retskaffen , " tcenkte han bevceget , ~ og gjpre hende lykkelig , — bcere hende paa Hcender og ikke altid tåge det saa n / sic med din Vcerdighed ; <b>Kvinderne</b> er nogle fine Skabninger , — de er svage , og vor Kraft bestaar deri , at ikke ogsaa vi er snage . " , Han traadte sagte ind , Tante og Niece kncelede i Vinduesfordybningen, og Maaneskinnet faldt smukt paa de to Skikkelser . jeg komme ind ? " spurgte han sagte . Han kncelte ned ved Siden af dem , og den moderlige <b>Kone</b> lagde de unges Hcender i hinanden og sagde : ~ la , saaledes skal eders Forlovelsesstand begynde , med foldede Hcender og <b>Blikket</b> mod Himlen . Herren fM eder , han vcere eders bedste Del ! Klinger end dette underligt for eder : — tror det ftrst , saa vil I faa erfare det . " O nei , det klang ikke underligt for dem ; Marie havde trods sin muntre Natur og sit tilsyneladende lette Sind , en varm <b>Kjcerlighed</b> til Herren dybt i sit <b>Hjerte</b> , og hvad var det vel , som i lang Tid havde draget den unge Mand hen til Forstmesterens stille , vcerdige <b>Kone</b> og til hendes Pleiedatter ? Et Tilknytningspuntt til den Fred , som overgaar al Forstand , fandtes sikkerligen hos ham , en Traad , som havde sit Foeste udenfor den forgjcengeligc Verden . Herren <b>selv</b> vilde spinde Traaden videre og arbeidede der- Paa ogsaa i denne Time .

<h4>396</h4>. mener , " tog denne temmelig forleqen Ordet . at Kicerl.ghedens Maal netop er en gjensidig Formdllnq " Fetteren vilde videre , men et <b>Blik</b> fra Tanten bragte ham til Taushed . Det var jo ingen Hemmelighd for nogen , at SchlMsers og Emilies Hjerter havde fun et hinanden . Enulie havde sikkert fM , hvad der manglede over hende , og Paa hvem hun lkke kunde ftnde noget at udscrtte . Generalindcn tog nu hurtigt <b>selv</b> Ordet : ~ Om Kicrrlighed gaar det ikke an at disputere , fordi saa mangen en NYgtig Mcl , e i Hjertet Fordring paa dette Navn " sagde hun , hvorpaa hun i Korthed forklarede de to Merrer Yvorom Samtalen havde dreiet sig . " ' " Af egen Erfaring kan jeg tWie , at jeg meget ofte MelMe mig , dengang jeg endnu gik paa Val , " sagde LMtnant von Reifenhagen , ~ jeg vidste aldrig hvorfor , men leg mcerkede altid , at det gik over . " ~ I det overfladiste Verdensliv kan der sjelden verre om en dyb og <b>sand</b> <b>Kjcerlighed</b> , " svarede Elisc . ~ Nei , faldt Tanten venlig ind , ~ men den ljcrre Gud vaager dog ofte saare barmhjertig over sine letsindige BFrn . " " Dette skal vcere en Hentydning , " sagde Generalen , min <b>Hustru</b> og jeg lcrrte hinanden at kjende paa et Bal , og forelskede os straks i hinanden . " ~ Det skal dog ikke vnre noget Forbilleoe for Ungdommen," tilftiede Generalinden smilende , , thi det faar Me ofte et saa godt Udfald . Jeg var ikke vcerd , at Herren fMe mig saa trofast , og det var ikke min Fortjeneste at ieg ftk en saa brav Mand ; de fleste af mine bekjendte , lom ikke var lechndigere end jeg , er blevne meget ulykkeliqe og har gaaet ganske tilgrunde i Vcrdenslivet . " ~ Vi to kan imiolertid allerbcdst modsige Emilies Paastand, ' sagde Generalen , jeg har dog virkelig forandrct mig as <b>Kjcrrlighed</b> til min <b>Hustru</b> . Vistnok mcerkcde icg snart , at jeg havde faaet en saa " — Generalinden kysfede sin gamle Mgteherres Haand , men lagde paa samme Tid Fingeren paa hans Mund . « Der kan man dog se , " tog Generalen Ordet efter denne Afbrydelse , ~ at jeg staar under Tøffelen . "

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011051320021'target=_>Rosenius, C.O., 1880, Fra Døden til Livet</a><h4>87</h4>N Undertiden har vi mcerket hans advarende Most og hans Aands milde Banken paa vore Hjerter , men hvad andet har vi g , ort end fornegtet dette altsammen , flyet tangere og lcenaere bort fra ham , levet et Lw t end stsrre Syndesikkerhed , og har under alt dette knap etOtebllk frygtet forHelvede . Hvad stal man vel kalde saadant andet end en jammerllg Sikkerhed . Men saaledes er jo hele VerdensLsb Saadan er alles vores Natur . Forundrer du dig derover saa tcrnk blot ester , hvorfra vi ere komne , hvorfra vi nedstamme . Det var jo Slanaens fsrste Ord til Eva : ~ Skulde Gud have sagt ? Skulde voere bundne afnogenLov ? Og : ~ I skulde ingenlunde do " . Se der har vi Gri.nden og Kilden . Da veg Guds Fryqt fra Mcnncstets Natur , og frasen Stund er den aldrig sunden der . Ncerk : vi tale om Naturen - ikke om hvad Aanden kan virke i os . Vi have ikke enqana Evne til at mose Wcenke oa erkjende denne almindeligc og aabenbare Ugudelighed og Slkterhed , ssrend Guds Aand faar va-kket os ssorst da lcrrer man at kjende Fordcervelsen . Da vil man frygte og elske Gud men kan ikke . Man vilde , smderknust og ydmyg , angre og bekjende Syn ' den , men man kan ikke ; man vilde scette sin Fortrostning til Herren , men nei , man formaar det ikke . Man a-ngstes , man forundrer sia herover man tror stg at verre den Eneste , som er saa uskikkelig , og som er stillet i saadan en Ulykke . Men der findes ingen Ting , der saaledes som denne er saa almlndelig , den er vor fcelles Arv fra Adam . Adam syndede forfcerdclmt mod Guds Bud . Alligevel flyede han bort fra Herren oa vilde ikte bchende sm Tvertimod undstyldte og forsvarte han sig , og han hy lede or Gud Forundre dlg da ikke over , at ogsaa dit <b>Hjerte</b> er ! i Hykleri oa al Ugudelighed . Menneskeh ertet blev ved Adams Fald saaledes beskaffent . og har siden den Tid aldna vaeret anderledes . Dette er jo en forstrlrkkelig Synd , efterdi den strider lige imod hele den forstc Tavle i Guds Lov . Du arbeider , du kjcemper , du ' crnqster dia du foriindrcr dm. Forundre dig dog ikke herover ! Adam begyndte strar efter Synoefaldet at styde Skylden for sit Fald over paa sin <b>Hustru</b> ; han chkcde hende ltke nu mere saaledes som sig <b>selv</b> . Der er Kilden - i ) u vil vcere ren fra alle onde Begjcerligheder , men du kan ikke : thi Eva saa paa Trcect det var lystellgt at se til og godt at crde af . Dette er vor Arv . Oa . denne mcdforer ikke blot <b>Hjcrtets</b> Ondskab og Begjcrrliahed men ogstia et Morke oa en Forvirring i Forstanden . Og deri liaqer din Nodog Du kan aldria ncrre en rigtig Tanke om Gud : du er Mr naar du burde frygte . og du frygter , naar du burde fryde dig med al Trost og Glcede Du kan saaledes as egen Kraft aldrig bevare det rette Billeoc af Gud , Du horer Evangeliet , du finder det <b>sandt</b> og herligt du vil indskrive det i dit <b>Hjerte</b> med en uudflettelia Skrift og doa lorsvmdcr det i Oieblikket - du hensynker uophsrlia atter oa atter i dig <b>selv</b> og i dine egne Gjcrninger . Gud bliver dig blot som Dommer , det er dia , som om vi ingen Frelser havde o . s . v. Forundre dig ikke ! Slangen sagde til Eva : Oud ved , at naar I cede af denne Frugt , stulle b ' ve ligesom Gud : men det under han eder ikke ; han er ikke eder en laa huld Fader som I kave troet . Denne Tale forstyrrede aldeles det rc cßlllede af Gud f 5 e fsrjte Mennesters Hjerter faa at der . som Luther siger , stlllede sig frem for deres <b>Oine</b> et . . Spsgelse med gyselige

<h4>351</h4>tens Blod ? ( Ebr . 10. 28 f . ) Her ser du . hvad Vantrocn er i sig <b>selv</b> . Men derhos er den en fruktbar <b>Moder</b> til alle andre Synder . Nen . som ikke tror paa Kristus , er bortuendt fra Gud , har ikte nogen <b>sand</b> <b>Kjerlighcd</b> eller Lyst til ham og hans Vilje , <b>selv</b> da naar han af Trceldomsfrygt eller i falsk Indbildning om egen Fortjeneste lyder hans Ord paa den Vis , som Hyklernc og Gjerinna / ohellige gjor det . . . Saaledes folger " . som Luther siger . . . Djauclens Dragehale og hele Helvcoe med Vantroen " . Doq dette er ikte den egentlige Grund , hvorfm Vantroen bringer Fordommelsc , men Grlinden dertil er den , at den forkastcr Guds Naade . og at for den . ikte tror paa Kristus , er der ikke mere uoget Offer for Synderne , — han staar der nsgcn for Gnd l sine Synder . Men her knnde en sporgc : Naar Aandcu lun stal aabenbare Vautrocus og ingen Synder med Loven tan fordomme os . hvorfor taler da Gnds Ord pa meget om Synd nnod Guds Bud ? Svar : Som allercdc for vist . tales her om Aandcns cgoitlige og fornemste Gjcruing . Synder mod Loven tan ogfaa Tamvittigheden og Fornuften straffe ; men kun Aanden tan aabeubare . Hvad Loven i Almiudclighcd skal tjcne til . er udtalt af Apostelen Paulus i Gal . 3. Den blev foiet til for Overtrcedelsers Skyld , d . e . forst for at styre og afholde de vantro fra grove syndige og dcrncrst for at vcerc en Rettesnor for alle Mennester . hvoraf de kunne vide . hvad der bchaqcr eller nushagcr Gud . Men hvad Loven skal ndrctte i Omvcndelscns Verk og derpaa i den sande Bedring . lidtales i Rom 5. Efterat Apostelen der forst har framstillet den store artlkel . at ligesom ved et Mcnneskcs Ulydighed mynden og Doden er lommen over os , saa kommer ogsaa ved et Menuestes Lydighcd Liv og Rctfoerdighed , tilfoicr han l V . 20 : Men Loven kom til . paa det at Faldet skulde fremtrcede dcs overflodlgere . Men hvor Syn de n er bleven overflodig.dererNaadenblevenendoverflodigere . Her hlcelper intet andet . Mennesket lcerer aldrig at fly til Frelseren og antage ham . medmindre det jages og tugtes af Syndcn og Loven . Gud er forsonet . itke lcrngcre vrcd . hans barmhjertige <b>Hjerte</b> brcrnder af <b>Kjcerlighed</b> til alle hans dyretjobte ; men de kunne ikke hjcelpes. kunne ikke formaaes til at fly til . . Fristccdernc " . uden at de blive drcvnc af . . Blodhcvncrcn " . Derfor maa Gnd altid plage , forfcerde . ia lldmattc os med Lovens Bud oa , Domme . Josef brcrnote af <b>Kjcerlighcd</b> til sine Brodre . der de kom til Mgypten . og hau var strax bestemt paa at gjore dem godt ; mcv alligevel talte han haardt til dem , lod dem binde og fcrngste , ja forfcerdede og forstrcekkedc dem . for at boie og ydmyge deres haarde Hjerter . Saaledes maa ogsaa Herren ved sin Tjeuer og Tolk , Moses , forfcrrde . fauge . tviuge og bedrove os . Men han . . plager ikte a f sitHjer te Mennestens Vom " sßegr . Vog . 3 ) . Alt hvad Loven siger , siger den . for at huer Muud skalti lstoppcs. ogalVcrdcnskal vccre skyldig for Guds Dom ( Rom , 3 ) , — at Syndcnskal blive des overflodigere . Hvor- dette gaar til . sces af Rom 7. Loven viser os ikte blot Synden som i et Speil , men den vcekker ved Forbudct dcv slumrcnde Synd , for at den itke lcrngerc stal ligge der dulgt og skjult , og Syuderen ikke lcenvcere

<h4>516</h4>sormanede dem , samt kaldte dem sine ~ Brodre " . Men de vare saadanne. som ikte blot sogte en tilfcrldig Aflosning . for siden at gaa bort og folge Syndcns ogVcrdcns Vci igjen , men som frcmdcles vllde blive hans egne , blive og leve under ham i haus Rige . Mcerk , de bleve hans Venner og Eftcrfolgerc for hele sin Tid samt dlcve under hans Veiledmng mere og mere fornhcdc og hclligcde i <b>Sandheden</b> . De blevc derfor en sta den almmdclige Masscs Vci afvigcndc Hob og taldtes derfor . . Partl " . de . . Nazarcrcrcs Parti " , som bcspottcdcs og forfulgtes af den store hob . hvilkct de led mcd Lyst og Glcede . Saadanne blcve de , som annammcde Jesus , og . . alledcm gav han Magt at vorde Guds Born " , ia alle . alle , som annammede ham . Historien ncevner itke en eneste , sum sogte Naade hos ham . som blcv afvist som formeget syndig og uvcerdig , Samaritancrindcn , saa hedcnst og lastcfuld hun var — hun havde havt fem Mcend , oc ; den . hun nu havde , var ikte hcndes — sik stiax Naade . og den batmhicrtige som hungrig og trott af Neiden havde sendt sine Disciplc at kjobc Mad , bchovcdenuikte at splse . jaa mcettct var han ester Syndcrcs Frelse hnngrcnde <b>Hjerte</b> ( Joh , 4 ) . Simon . lonce Son , nedfaldcr forsircekkct og raabcr : ~ Gaa ud fra mlg , thl zcg er en syndig Mand ! " og faar strax det trostellge Svar : . . Frygt ltte . fra nu af stal du fange Mcuncskcr ! " — Den Voerkbrudne naaedc lttc til at aabne sin Muud og bede , forend Jesus raabte linud ham ! ~ Vccr frimodig , min Son , dine Synder ere dlg forladte!" — Salllus var en . . Vcspottcr og Forfolger " , som for omkring og dvcrbte Gnds Born ; men han sik strax Barmhiertighed , saa at han endog blev den storstc Apostel . Korteliq : . . Alle oem . som ham annammedc , gav han Magt at blive Guds Horn . " Heraf se vi nu , hvad det er at annamme ham . Ogsaa l vore Dage gaar det ligcdan til . Du ser jo den store almmdclige . sorglose Hob , som formcdclst sit Kjobmandstab . sin Agcr . sin <b>Hustru</b> ikte har Tid tll at agte paa Indbydclscn til Lammets Oryllup . De bckjende Kristus i Kirken , de erc kaldte efter hans Navn , — de feire hans Fodjelsfest . hans Lidelses- og Dodsfest , — de cede og drikke med ham o . s . v. , men sorovrigt ville de ikke have noget med ham at gjore ; men leve hele sin Tld tilfreds forudeu ham i Forfcrngelighcd og Synd . — og de stulle ~ do t deres Synder " , blot fordi de lkke „ hylde Sonnen " . Men du ser ogsaa en liden Hob arme Syndcre . hvis hele Liv er en Sogcn efter Jesus. De ere ellers ikke fuldkomne Mennestcr , de sukke og klage <b>selv</b> daglig over sine Synder og Skrobclighedcr ; men Et udmcerter dem ; De tunne ikke uudvcere og hans Naade ; deres Tanter og Sukke ere disse : ~ 0 , om zeg vidste , at ieg var et Guds Barn og havde mine Synders ; o , at jeg kunde rettelig tro og folge Frelseren!" o . s . v. Se , disse ere de , som annamme ham og tro paa hans Navn . Er ogsaa du en af dem , da har han ogsaa givet di a Magt at blive Guds Barn . Om du end ikke altid kan tro med fuld Vished og Glcede . — naar din sogende Sjcrls <b>Blik</b> dog er rettetpaa den Ophoiede med Lcengsel og Onste om at kunne tro paa ham og paa ham alene , da er dette vissclig at annamme ham . som vi jo tydelig se , at de , som kom til Jesus , ikke tom med Fryd og Fred . men kom snarere grcedende , sukkende , bedende , - — og se ! — til saadanne sagde dog Herren : ~ Din Tro har frelst dig ! " Hvori viste deres Tro sig ? Snndelig ikke ien fuld

<h4>558</h4>at alt kommer fm din Gud . og du elsker ingenting andet end ham og hans Vclbehag Hvor er un det helligc Mcuneske , som holder dette Vud ? Vi ville nu tale med en saadan . Er du lige rolig og lytlelig , om nogen tager al din Eiendom fra dig og du nedsyukcr i Fattigdom ug Mangel ? — lige rolig og lyktclig , om uogcn bcrovcr dig dit gode 3 iavu og om du bliver oerclos , foragtct og afskyet af alle Menurster for hele dit cfterfolgcnde Liv ? — lige rulig og lyklelig , om en svcer Sygdom . ja en Morde.r forkortar dit Lw ? — Om det er <b>sandt</b> , at du elsier Gud af dit ganste <b>Hjerte</b> , af dit ganske Sivd oa af alle dine og dcrhostror ftaa ham alene , faa at du ved , at ingenting <b>moder</b> dig uden hans Vilje , jaa maa du nodvendlg vccrc lige rulig og lyttelig ved alle Ha ' udelser . Men maastc det er faalangt derfrå , at du tvertimod blivcr foiuroligct over et ganste ringe Tab — at om du blot faar at vide , hvorledes mau har talt ilde om dig eller blottet nogen af dine Svagheder , det forftyrrer din No for hele Timer og Dage ! Ia maaste blot et foragteligt <b>Blik</b> foruroliger dig ! Hvorledes elsker du da Gud alcuc og hans Velbehag ? Ia fuler du , at du rigtig varmt elsler din Gud . saa at dine Tanter cre bestandig hos ham ? Maasfe du tvcrtlmod megct livligcrc elsker og tccnkcr paa et Menneske ! — Men vldere Er det ilke <b>sandt</b> , at vi , som ere gjcnlostc med Guds Sous Blod sra alle vore Syuder , fra Dodeu og fra Djcrvelcns Vold , burde intet hoiere Maal have sur vort Liv end at forhcrliae ham , som er dod og opstanden for 05 ? Er dette nu virkelig til alle Tider din Strcrbcn ? — Er det ilke <b>sandt</b> . at . om du elsker Gud af dit ganste <b>Hjerte</b> . da stal du ikte kunue trivcs saaledes ved nogen Ting som i fortrolige Bonncr og Samtaler mod din Gud ? Har ' du det virtelig saaledes. at du bestaudig vil omgaas med ham i Vounen ? Maaske det er saalaugt derfrå , at du med Gud i Vov ! At . hvorledev er det da med dit Forhold til det wrste og storste Vnd ! Deruast skulde du-ogsaa . . elske dm Nccste som dig <b>sclv</b> " . Tcrnk altid alvorligt paa disse de storstc Bud fra Herreu din Gud . Hvad er al um vi itte holde de vigtigste Aud ? Er det virkeligSand- at du hav ligcsaa Omsorg for din Na , ' stes Bedste forn for dit egtt ? Acta ' , ! k . at med din . . Ncrfte " mencz ikle blot en og anden men alle Men nesler . Venner og Uvenner . Er du liacsaa ivrig for ethvert andet Meuneskcs Velferd sum din egen ? — ligesaa om for et dadlende Ord om div Ncestc . som uaar du horer , at man har talt ildc o : n Vidcre : Tv tvor . at euhver , som dor uomvendt bliver evig foidomt , Dersom du da elsker div Ncrstc som dig <b>sclv</b> saa bor du jo anveude samme Flid . Iver og Nidkjcrrhed for rl ' , oert andet som om det g , aldt div egcnSalighcd ellerFurdoiumclie umagcr dig fur uogie faa ; men maaske du daqliq ser maugc uonunudtc . for hvls Va ' ttelse du ikle gjor dig miudste Moie ! er dtt da med div <b>Kjcrrlighed</b> til dalten ? Og hvorledcs ser det ud med alle dc ovrige Bnds Efterlevclse ? Er det ikte <b>sandt</b> at Tmg rore sig i dit ved en ganske ringe Anlcdmug jum Gud Herren hadcr og furoommer . faasom Vrcde vad . Hoffcerdlghcd . Egenkja ' rlighed . urene Lyster og Beqjcerlighed ester andres Eieudom m . m . Og vi tale dog endnll om troende

<h4>572</h4>leve for Gud . ikte kan bcere Gud Frugt , uden at man fsrst er dyd fra Loven . Vetragt tyende Forhold — forst det , som Erfaringcn lcercr : Saalcenge en Sjcel ikke er frigjort fra Trceldomsaandcn , men arbeider paa sin egen Forbedrina med ' <b>Oiet</b> rettet paa sia <b>selv</b> og paa Loven , men ser Gud alene som oen , der fordrer noget as ham , saa er alting dodt . koldt , tungt og modbydeligt i Hjertet , og med alle Forscetter blivcr der ingen Kraft , ingen Udfsrelse , men blot Tanter og Ord og nogle udvortes lagttagelser , som ikke ere for svcere for Naturen . Vidcre , om Loven angriber Hjertet og den indre Fordoervelse , saa bliver denne oprort og opegget til dobbelt Kraft , som Apostelen siger : ~ Da Vudet kom , blev Synden levende oa virkede al Begjcerlighed i mig " m . m O , hvormanae cengstclige Trcelle sukke ikke under denne Erfaring og kunne ikke forståa , hvori Feilen ligger ! De hore og lcese Guds Ord , de raabe og bede , de gaa i Skrifte , og alt er forfcengcligt , alt er idel Dod og Afmagt ; kun Synden er levende og mcegtig . O , at de vilde boie sit Ore til Gnds Visdom , ~ da skulde deres Fred blive som Vandstromme og deres Retfcerdighed som Havets Bolger , " da skulle de faa en ny , himmelsk Lyst og Kraft i sine Hjerter ! — Feilen er , at du aldrig er bleven ret frigjort fra Trceldomsaaget , aldrig ret har smaat , hvao den sande Tro vil sige , aldrig er blcven ret salig blot af fri , uforskyldt Naade . Om du fik det Lys , som ikke en Broder kan give den anden , men som er en Guds Aabenbarelse i Sjcrlen , en himmelsk Gave , det Lys , som forklarcdc lesum saaledes , at dn saa den undsigelige Naade, hvorvcd den himmelske Fader , under hvis Bud du trceller , har saaledes forbarmct sig over os , at han har hcngivct sin Son til at opwlde for os alt , hvad Loven nogcnsinde krcevcr af os , at han ogsaa for dig alle redc har ndfort alt , hvad du nu trcelle r for , og at Gud nu intet andet vil mcd sin Lov end ndmatte og drccbc din Sclvvetfcrrdighcd. cftcrdi han vel vidste , at du ikke formaar at gjore andet end synde ( Rom . 3 , 12 — 19 ) ; men han er forsonet for al d ' iv Urctfcerdighcd. saadan som du er , elsker die ; just som du er , har <b>selv</b> borttaget alle dine Synder og tcguct dig i just som du er , for Kristi , diil Brodcrs og Midlcrs Skyld , — om du i Aandcns Lys fik <b>Oie</b> paa dette og saa , hvorlcdes den himmelske Fader aldrig gjor Regnina paa dig , men har alt og nok i sin elskeligc Son , for hvls Skyld han aldrig , aldrig ser ftaa dine Synder , ja , om du saa , hvorlcdcs du midt under din dybcstc Ncdslagcnhed over Synden er tcrkkclig og clskclig for Gnds Oinc i Kristus , saadan som du er , — om du i Aandens Lys saa dette , saudclig . du skulde blot grcede af <b>Kjcerlighcd</b> og sftringc af Glcedc , og alt , hvad der hidtil har vccrct dig altfor sucert og umuligt , skulde blive dig let , lifligt og clskcligt . Hvad var det , som gjorde , at Martyrcrne gik mcd Fryd til Vaalct , at Vincentins kaldte t > e glodnde Kul Roser , da han stcgtcs . at svage <b>Kvindcr</b> sagde , at de aldrig havde gaaet til Dandscn nicd en saadan Glcede som nn til Baalet ? — Ingenting andet, end at dc vare fulde af den usigelige store Naade , at Guds Sem var deres Broder , og at de ved ham vare Guds Venner og hans Oincs Lyst . Se , dette er , hvad Skriften faa udtrykkelig siger : ~ Gla ? de i Herrcu skal verre cdcrs Styrke . " Og : ~ Naar du troster mit <b>Hjerte</b> , saa lover jeg dine Budords Vci . "

<h4>574</h4>Dette er det forste Forhold , hvorvcd vi forståa , at vi maa vcerc friedefra Loven for at kunne bcerc Glid Frugt . Det andet er , at , om dn cud kunde gjore mcgct mere godt end alle troende saa er ikte en cneste af dine Gjernmger Gud bchagclig , saalccnssc de ikte flyde af den evangeliske Tro og <b>Kjcerlighcd</b> . Saalccngc Loven herster i Samvittighedcn . saa duer ikte noget af alt det . du gjor . Dn gjor maaste aldeles samme Gjerningcr som den troende og Nigzorte Sjcrl ; men paa dine Gjerningcr er en styg sort Plet . som gjor dem rentud modbydelige for Guds Vine . Denne sorte Plet bestaar blotden . at du <b>mor</b> det Gode for Love us Skyld og ikte af Hjcrtcts egen frie Lyst og Kicerlighed , - at du gjor det Gode sorßnds , sor Trnslers og Loftcrs Skyld og ikte gjor det af dig <b>selv</b> , af fri Hv og Lyst , — ligesom det scrttcr en Plet paa Barnets Venlighcd mod sin <b>Moder</b> , at det er blevet tvuuget dertil vedFadcrcus Nis eller lokket dertil ved et Lofte , da det jo burde bevise denne Vcnlighed af sit eget <b>Hjcrtcs</b> <b>Kjcerlighcd</b> til sin <b>Moder</b> . Nu er det af Guds Ord aabenbart . at Herren ikte vil have nogen tvungen Tjcnestc : han vil egcntlig have Hjertet . detsKjccrliqhed og Lyst . Ligesom en Mgtcmand , der rigtig clslcr sin <b>Hustru</b> , ikte er tilfreds med ' blot at faa Tjcucstcr af hende , men vil allcrforst eie hcndcs <b>Kjcerlighcd</b> . og om han ovdagcr , at hnn blot for hansEicndons Skyld anstillcr sig , som hun clstcdc ham , men ilke : <b>Sandhcd</b> gjor det , da maa jo al hcndcs Vcnlighcd blive ham umumclig og tun opvcetkc hans Vrede , — jnst saaledes er det ogsaa mcd Gud . For haus <b>Oine</b> er al den Fromhcd og Gudfrygtighcd va-mmclig og forkastclig , som ovcs blot fordi man skal , man bo r. man maa saaledes , d . e . ovcs blotaflovmcessia Bevcegqrnnd og ilke flydcr af en frigjort , salig og takncmmclig Aand , af ' Hvudcns ztjccrliglicd , som siger saaledes : ~ ) iu ' gjor jeg ikke Mmcget som at optagc et for at lighcdcn og Guds Vcnstab eller sone for min thi alt dette har min Blodbrndgom gjort , da han i tvcdivc Aar trccllcde for mig nndcr Loven og sidcii led Korsctv blodige Dod ; mcv det Lidct , jeg gjor , det gjor jeg nctov , fordi zcg ingenting bchoucratgjorcformin Vcnaadning , inst fordi han har gjort alt for mig og er <b>sclu</b> til cnhvcr Tid min Rctfcrrdighcd for Gnd — alcnc fordi jeg nu er i et saadant Naadcrigc , at alle mine Syudcr aldrig tilvcgncc ' mig , at jeg aldrig mere skal dommcs efter Loven , Just denne Frihed bcvcrgcr mig saa inderligt til gjerne at gjorc og lide alt , hvad Glid vil , cftcrdi hau har tagct mine tccrendc Hjcrtcsorgcr over Syndcn og Loven bort fra mig . la , da han har tagct den store og tniigc Vyrde , min ftaa sig , da vil jeg saa gjerne berre lidt paa disse lcttcrcVyrdcr , og forstigc Vcrdcn Syndcn , ilndcrkue mit og tjcnc min Ncestc i itjcerliahcd mcd Gjcrningcr , Ord og Taalmodighed . give den hungrige Brod , dcnHnsvildcHcrbcrg , bcsoge dcv Sygc eller Fangnc , tilgivc min Uven og have Taalmodighed mcd et eller andet biydsomt Menneske eller et bcsvcrrligt Kald i Llvet . Eftcrdi min Herre ogsaa har vcerct saa naadig at ville lade saadanne Smaatiug behage sig , saa er jeg ucestcu ligesom halvt i Paradis over , at jeg maa og kav gjorc dcv trosastc , dyrebare Herre noget , som behager ham . dette er mig hjcrtclig tjcert , sordi han er saa ovcrmaadegod og har oprettct dette Naadcrigc , at ingen Synd tilregnes mig , at jeg aldrig stal dommcs efter Loven . "

<h4>578</h4>billiger den og giver Gud Ret i hans Ord , at i os er idel Synd og Afmagt og i Kristo alene baade vor Rctfccrdighcd og Styrke . Han efter at ' kunne gribe og kjende Kristum og hans Opstandelses Kraft og ved , at da er alt vel ; hvad han her lider , det lider han af Svaghed . Kans Sind og Forscet er saaledes evangelisk , det er blot Fslelsen og Naturen , der ere lovagtiqe . Og stjont han da maa ligesom tvinge sig til det Gode under Folclscn af ' Kjodcts Mudstand , saa er det dog den store Naade , som han tror ftaa , der bcvcegcr ham til saaledes at ville tvinge sit gjenstridige Kjod . Han elsker Herrens Vud , endog hansTrusler, ja hans Ris og Tugtclse , blot fordi de bidrage tilKjodetsDsdelse ; men just denne <b>Kjcerlighcd</b> til Loven og Truslcrne har kan deraf , at han tror oen store , uforskyldte Naade , og saaledes ere jo da hanspjerte<)rund og Bevceggrund evangeliske . — Helt anderledes er det , omman blwer . . fangen unoer Trceldommcns Aag " , naarTankerne fsrst og sidst ere rettede paa us <b>selv</b> . og hvad v i skulle vcere og gjore . hvorved Barnetilliden kvoeles , og Loven bliver rcntnd den herskendeMagtiSamvittighcden. Da indtrcrder atter det Syndevcescn og den Afmagt . som Loven altid virker . Og heraf se vi da , hvor nodvendig Samvittighedens Frihed fra Loven er ogsaa for den sande Helliggjsrelse . O , ak hver tnsten derfor i alle sine Levedage tog Apostelens Ord vel til <b>Hjerte</b> : . . Vliver stadige i den Frihed . hvoriued Kristus frigjorde os , og lader eder ikke atter tvinge Trcel ) l . , n - . ! ens Aaa ! " ( Gal . 5 ) . Saa siger og vor Herre Kristus <b>selv</b> om denne Eag : . . Bliver i min Kjcerligheo ! Hliver i mig ! Ligesom Grenen ikke kan bcere Frugt af fig <b>selv</b> , uden den bliver iTrcect , saaledes kuune I ikke heller , uden I blive i mig . " ~ la , " siger du , ~ jeg har set herlige Ord og Beviser for den store trssterige <b>Sandhed</b> , at vi , som ere fortabte for Loven , men holde os til Kristus , ikke skulle dommcs ester Loven , men vcere saa fri fra dens Domme , som <b>Kvinden</b> er fri fra Mandcns Lov , naar denne er dod og begraven j jeg har ogsaa set , hvor aldeles nodvendig denne Frihed er , itke blot for at tuune forblive i Guds Naade , men ogsaa for oen sande HelliHgjorelse : men det er mig dog aldeles nmuligt at kunne rigtig tro og fastholde denne Frihed formincgen Tel ; thidels er det altfor stort og herligt , at jeg arme Synder stal leve i en saa bcstandig og urokkelig Naade hos Gud , som om ingen Lov eller Synd var til ; og dels fole r jeg ikke det mindstc af en saadan Naade og Frihed , men foler tvertimod det modsatte , nemlig at jeg endnu idclig anllagcs og dommes afLovcu . Hvorledes kan jeg oa tru . at jeg er aldeles fri fra allcLouens Dumme , at Gud elster mig og anscr mig , som jeg aldrig syndcdc , som om ingen Lov og derfor ingen Synd var til , men ' idcl 3 tcnhcd . Rctfcerdighcd og Velbebag for hans <b>Oine</b> , — og alt dette blot for Kristi Skyld ? Nei ' , det er for stort , for herligt , ugjcg folcr jo Syndcn ogLouens Dommc over mig . " — Dette er en dyb Indvcndiug , vel itkc i sin Tanke , men i sin Rod ; thi den sidder i sclvc vor thi Lovbevidsthcden og Lovbegreberne ere visselig ved Faldct sordunklcoc . men ligge dog saa dybt indftrentcde i al Menneskenatur , at cudogsaa Hcduingcrnc , som aldrig have hsrt et Ord fra Sinai , dog plage sig tildode i deres Indre ( Rom . 2,14 . 15 ) , og derncrst blcv det ved Syudcfaldet vor bestemte Natur ikke at tro ogagtc Gud , hans Ord ogGjcrningcr . men tro os <b>selv</b> , vort Tykte og Fslelse . Dette gjsr , at <b>selv</b> de mest euange-

<h4>584</h4>den helllge Apostel har gjengivet det her anfsrte prsfetiste Stcd 40 , 7 — B mcd <b>Oiet</b> rettet paa Ofrenes Modbillede , nemliq Kristi Legeme . skulde hcngives til et Forsoningsoffer for vore Synder Men i det anfsrte Sted af Salmen findes noget , som er mcerkvcerdiat for vort ncervcrrende Emne . Der staar nemlig : Du har ajennemboret mig Orene , Udtrytket sikter tydelig til et Sted i den levitifke Lov , der handlcr om saadanne Tzenere . der ville forblive i sin Herres Tjeneste for hele Livet ; da ikulde de mcerkes dermed , at deres Oren ajennemboredes. Dette Lovbud lceses i 2 Mos . 21. og lyder saaledes : Hersom siger hart : leg elsser min Herre , min <b>Hustru</b> oa. mine Born . og vil lkke gaa fri ud . da skal hans Herre fore ham frem for Dommeren og holde ham op til Dsren eller Dorstolpen , og hans Herre skal aiennemjtuige hans Ore mcd en Syl . saa stal han tjene ham evindeliq . " Oa dette har nu Kristus i Profetien anvendt paa sig og sagt til sin Fader- har du gjennemboret mig ; ieg vil ikke verre fri . zeg vil vcere din Wiener ; zea stal ajore din Vilje , o Gud . mit Leaeme stal verre det Offer , som du med alle disse Ofre hartilsigtet ; der er strevet om mia ißoaen . ? t leg stal din Vilje , o Gud ! " Oa da tillcegaer Apostelen : Sed hvllken Vllze yl ere helliggjorte formedclst Jesu Kristi Legems Opoftelse den eve Gang , " » > Issen i vor Tert staar . at Guds Son ligefra Fsdselen af var stillet under Loven : . . Gud sendte sin Son . fsdt af en <b>Kvinde</b> . fsdt under Loven ( lfolge Grnndtexten „ vorden af en <b>Kvinde</b> . vorden under Loven ) paa - ? ? " " wilde frikjsbe dem . som vare under Loven . " Han skulde saaledes t hele sit Liv verre under Loven , og det sandelig ikke for sin egen Regninq men paa det han skulde frikjobe dem . som vare under Loven . " Ligefra den ottende Dag . da han ifolge Loven blcv omstaaren . var hele hans Llv en stadig Lovopfyldelse — for os . Loven krcever . at vi stulle elske Gud af vort ganske H erte . af vor ganske Sjcel oa af vort ganske Sind og vor Naeste som os <b>selv</b> . men ingen af os ovfylder den ; da kom Kri'l"s oggjorde det for os ; han elstede Gud af sit ganske <b>Hjerte</b> oa af sin ganstc Sjcel ; det var hans Mad at gjore Faderens Vilje , og han elstede sm Ncrste som sig <b>selv</b> . han hengav sit Liv for Vrodrene . ja for sine Uvenner . Og Apostelen siger udtrykkeligt . at alt dette stcte . . for os " stete ~ for at frikjobe dem , som vare under Loven . " Aetcenk alt dette vel ' saa stal du med stor og salig Forundring finde . at vi aldrig behove at holde Loven for derved at forhvcrve Guds Naade og Saligheden men at den Sag er af Guds store Barmhjertighcd aldeles lagt paa en anden Mand . paa vor Midler og Lovoftfylder . Saa elstedc Gud Vcrden . at han hcngav sin Son den enbaarne . Syndefaldet var saa grundforstyrrcnde for alle vore Evner , at alt i os er forgiftet . opfyldt af Synd oa Ondskab . saa at ikke et eneste Vcesen af hele Slcegten kan opfylde Herrens Lov . men alt. hvad der er i os . strider imod den i alle dens Bud , hvilket de. som ere blevne berorte af Herrens Hellighed , dybt og bittert fsle naar de arbeide og kjcemfte for at oftfylde Loven . Og da nu Gud for sin evige <b>Sandheds</b> og Retfcerdigheds Skyld ikke kunde ' eftergive et Bogstad eller en Prik af Loven , hvorfor alt Kjod laa under evig Forbandelse . ) om v ' , ogsaa dagliaen fsle . — saa fattede Gud , dreven af sin uforanderlige Barmhjerttghed og <b>Kjcerlighed</b> til Mennestet . den naadefulde Beslutning. at sende sin egen Ssn til at opfylde Loven for os . Saadant

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009031612001'target=_>Lous, M.C. Theodor, 1865, Haandbog i den christelige Børnelærdom for Confirmander og Confirmerede</a><h4>262</h4>Huus , at virke til vort <b>Hjertes</b> og vort hele Vesens Helligelse, da vil Herren <b>selv</b> legge sin Velsignelse til vor Gjerning og udrette i os , at vi ved hans Naade vinde Kraft til allerede paa Hviledagen at begynde og gjennem Ugens svrige Dage fortfette forn fornyede i vor Einds Aand at ferdes i <b>sand</b> Christelighed blcmdt vore Bredre , med Sngtmodighed, Tnnlmodighed , <b>Kjerlighed</b> og Forligelighed , stettende de Vaklende , strebende at reise de Faldne , treskende de Bedrsvedc , opmuntrende de Forsagte , kort i Alt og blcmdt Alle i en <b>sand</b> christelig Vandel at aabenbare den udvortes Gudsdyrkelfes bedste og stjenneste Deel i de Dyder og sande christelige gode Gjerninger , som vi , drevne af Gnds Aand , udeve i det daglige Liv , lac . 1. 22 , 27. Frugten for os af en faadan Helligrlfe af Hviledagen formedelst Guds Ord , vil da blive en klarere Indsigt i Guds Rige , Kraft til en from Vandel og deraf folgende sterre Tillid og Fortrestning til Herren . Det er en Eelvfelge , ut den Christne , der som Familiefader har Tjenere sg Born , eller som Overordnet i andre Livsforholde har Underordnede , der skylde ham Lydighed , med Hensyn til det tredie Bud har den Pligt , ikke alene ikke at hindre disse sine Undergivne i at benytte Hviledagen efter dens Bestemmelse ; men endog meget mere ved eget Exempel , ved Veiledning , Raad og Formaning, om det skulde fornediges , at bidrage til , at den i <b>Sandhed</b> kan blive ogfaa dem en Helligdag , 3 : en Helligelsesdag. Medchristne ! Hvor uendelig naadig har dog ikke Herren veret mod os , at han forundte os at fsdes og leve i et christent Land , hvor Indbydelfer til Herren vor Gud lyde til os ved hvert Skridt , vi gjere paa Livets Vei . Hvor vi vende vort <b>Blik</b> , meder det Skole ved Skole , Kirke ved Kirke , hvor Guds Ecmdheds Ord leres og predikes puurt og reent , hans Eacramenter forvaltes uforfalstet efter Herrens egen Indstiftelfe , faa Ingen kan vere ganske fremmed for Herrens Naadesforanstaltninger til Synderes Frelse , hver Eendag og Helligdag kalder Klokkeklangen fra Herrens Templer, og indbyder hver Enkelt at trede ind i Herrens Huus for at opbygges , at treftes og styrkes ved det Ord , der er en Guds Kraft til Ealiggjerelse for hver den , forn troer . Men lader os dog ikke glemme , mine Venner , at alt dette dog ikke gavner os noget , fnnfremt vi ikke ville lytte til

<h4>491</h4>bliver den syndig , Eph . 4 , 26. , Matth . 5,23 — 25. 3 ) Misundelse, der itte under Nesten noget Godt , men heller gleder sig ved og gjerne enster at fremme hans Ulykte , lac . 3 , 16. 4 ) Übarmhjertighet » , der itte rores ved Nestens Ned og Jammer , lac . 2 , 13. Sverge vi nu : hvorledes stulle vi bedst tunne undgaae paa nogen af de ovenfor omtalte Muader ut forsynde os mod vor Neste , da er Svaret : Ved at frygte og elste Gud , saa vi vogte os for at treuke og stade hans stjenneste og herligste E kabling paa Jorden ; ved at beflitte os paa en <b>sand</b> og virkfom <b>Kjerlighed</b> mod Nesten ( Rom . 13 , 10 ) , og ved at vi stedse tenke os i Nestens Sted og sverge os felv : hvorledes vilde nu du , om du var i hans Sted , helst enste dig imodekommet ? Medchristne ! Vi gyse , nåar vi here tale om Mord og Drav ; vort <b>Hjerte</b> krymper sig af Forferdelfe , nåar vi fee den nlyttelige Morder , og vi ere faa tilbeielige til at mene , faa dybt ere vi dog itte faldne . O , lader os dog ikke bedrage os felv , mine Venner ; lader os dog itte glemme , det er itte Hacmden blot , men Hjertet , som gjer os til Mordere; thi Herren har jo sagt : Hvo som hader sin Broder , er en Manddraver , og hvad hjelper det os vel , ut Menneskene ikke hende os som Mordere , nåar Herren i Himmelen demmer os som saadanne ? Derfor , min Broder ! vil du preve dig felv efter dette Bud , da fee itte blot til hvud Huande » har gjort , men vend dit <b>Blik</b> indad i Hertets indcrste Grund og fperg dig <b>selv</b> , som for Guds Aafyn : hvad er det , forn har boet og boer derinde ? Har aldrig Hadets og Hevngjerrighetens vilde Flamme drevet Broderherligheden ud ? Har aldrig Vredens Kulde stuttet Kjcerlighedeus Ild ? Har aldrig Bitterhet ) , Hidsighed , Sheldsord faaret og bedrevet din Nestes <b>Hjerte</b> ? Og hvo blandt os Alle maa itte da beskjemmet vende Bliktet mod Jorden og betjente: Ak , Herre ! der fattes mig fnnmeget ogfaa i dette Stytte i at kunne bestane for dig ! In , i Eandhed mine Venner , Adams Eed boer i os Alle , og vi trenge Alle til at bede : Herre vor Gud ! tend du <b>selv</b> Kjerlighcdens Flamme i vore Hjerter , at den man drive Hnd og Fiendstnb , Hnardhed og Bitterhet » nt » nf vort Vesen ; lad din <b>Kjerlighed</b> varme os saa , at vi oprigtig lere at elste ham , der lod sit Hjerteblod ftyde for vore Synder og <b>selv</b> pnn Korset

<h4>611</h4>i dette , det vigtigste af Alle , her maa Religion og Bon vere vor daglige Gjest , og fremfor Alt bede vi daglig , at Herren ret vil aabne vor Forstand og vort <b>Hjerte</b> , faa vi ved hans Hjelp og Naade kunne blive dygtige til at fore et egtestabeligt Liv , over hvilket hans Velbehag tan hvile . <b>Hustru</b> , Bern , Familie , Huus og Hjem , hvormeget skjent , hvormeget elsteligt ligger der itte indbefattet i disse Ord , mine Venner ! de indbefatte i sig Forestillingen om de reneste , de edleste , de stjenneste Gleder , og alt dette har Herren vor Gud beredet os i den hellige Wgtestand , som han allerede indstiftede i Paradiset og gjorde til en Velsignelses, en Helligelsens Stand for vor hele Uddannelfe paa denne Jord ; o hjelp os da . Herre , at vi ogsaa stedfe og altid maa agte denne Stand og bruge den efter din herlige Hensigt , at din Naade fremdeles kan hvile derover ; lad <b>sand</b> <b>Kjerlighed</b> boe i vore Huse , lad Neisomhed , Tarvelighed og Orden smykke vore Boliger ; men fremfor Alt lad Gudsfrygt hellige vore Hjerter , at Fred og Enighed kan boe i Hytte som i Borg , at Bornene maa opdrages i Tugt og Wrbarhed, dig til Lov og Priis , dem <b>selv</b> til Fred og Salighet ) . Ja styrk os , Herre , at vi , <b>selv</b> rensede ved din Aand , kunne fere Reenhedens , Hellighedens og Kjerlighcdens Aand ind i vore Hufe og vore Familie , at de kunne blive din Helligacmds Templer , der , reiste paa Jorden , pege mod Himlen og i herlig Forening fore Mant » , <b>Hustru</b> og Bern ind til din Himmelborgs « forkrenkelige Seier . Det stee , Herre , for lefu Christi Skyld . Amen .

<h4>619</h4>om Utugt , da naturligviis Hjertets Reenhed er den ferste Betingelse for at bevare sig for Utugt , og Alt , hvad i det Foregaaente er sagt om disse Midle , har derfor ogfaa her sin sulte Anvendelse , faa at vi her blot behsve at nevne dem , nemlig : Maadehold i alle Nydelfer , navnlig i Mad og Dritte , Arbeidsomhet » og Flid , Afhold fra og Undgaaen af Alt , forn tan svette , eller udrydde Bluferdighet ) ? - og Ekamfuldhedsfslelfen , og fremfor Alt Aarvaagenhed og Ben , at vi maa frygte og elste Gud , og af denne Gudsfrygt og <b>Kjerlighed</b> hente Kraft og Styrte til at leve et kydst og tugtigt Levnet . Maatte dog Enhver iblandt os , ung forn gammel , Mant » og <b>Qvinde</b> , gift og ugift , itide aabne fine <b>Oine</b> og fee omkring sig i Verden , med hvilke sergelige , gruopvettende Trek har dog itte Herren strevet sit Advarselsord for os mod Vellystens og Utugtens fergelige Synd ! maatte dog de mange ulykkelige , de mange brudte Wgtestabe , de mange forfemte , ulykkelige Bern , altsammen Felger af Utugt og Vellyst , de mange Anglinge og Piger , der allerede i den unge Alder henstebe deres Liv forn udterede Oldinge , vansirede af de hesligste Sygdomme , en Gjenstand for Medmennestes Foragt og Medynk , lere os at forstaae , at Vellyst er en drevende Gift , al <b>sand</b> Lykkes Grav , at dens kortvarige Fryd fnart vil vende sig til navnles Jammer og Pine , at vi med Afsty man lutte vore Oren og Hjerter for dens lokkende, dens fristende Stemme , og i Felelsen nf vor egen Svnghed futte med David : stab i os . Herre , et reent <b>Hjerte</b> og fornye en stadig Aand inden i os ! lad din Aand boe i vore Hjerter , at ingen anden Ild maa brende demde end din Kjerligheds Ild , at vi , rensede ved din Naade , maa lere at korsfeste Kjedet med onde Begjerlighcder , og i Hjertets Reenhed tettes dig og engang indgaae i dit Samfund. Amen !

<h4>718</h4>fand Fuldkommengjorelse , <b>sand</b> Forbedring , som hnn i sit Hovmod itte feler Trang til , og stader derved sig <b>selv</b> uerstatteligt, ligefom han derved allerede i dette Liv bereder fig felv mangfoldige Vanskelighede og Übehageligheder . Istedetfor at vinde sine Medmenneskers Agtelse og <b>Kjerlighed</b> , stoder ham dem fra fig ; ved sin Ufordragelighet ) hindrer han den felles Eamvirten og tirre Andre til at legge Vind paa hans Admygelse , ja endog at unddrage ham den Wre , han elles kunde huve fortjent ; ligefom Stolthed og ifer Hovmod er den storste Anstedssteen for fund virkfom Broterherlighed . Skriftens Vitnesbyrd mod disfe Laster ere derfor ogsaa mangfoldige , saaledes foruten de allerede nevnte Steder , Philip . 2 , 3. , Rom . 14 , 10. , Salom.OrdsP . 27 , 2. , 29 , 23. , 1 Pet . 5 , 5. , o . fl. St . Af det faaledes Udviklede fremgaae altsaa , at den Christne med Rette agter Wen heit som et dyrebart Klenodie, som han streber at vinde og bevare som et serdeles ensteligt Middel til ctt virke til Gavn for sig <b>selv</b> og Andre ; men i denne sin Streben glemmer han aldrig Maalet for Midlet og med Guds Ord stadig for <b>Oie</b> vaage han opmerkfom paa sig felv , at han itte lader sig friste til nogen af de Feil , hvortil denne Streben efter Wre baade har fristet og vil fremdeles friste kun altfor Mange . Et kraftigt Middel til at verne sig <b>selv</b> derimod sinder han i Betragtningen us den blolte jordiske , den menneskelige Wres Forgjengelighet» og Ustadighet » , dens fuldkomne Utilstrekkelighet ) , og seger derfor forst og fremst eller rettere ene og alene sin Wre hos ham , fra hvem ene al fant » Wre har sin Rod og sit Ophav , hos sin Gud , hvem han frygte , elste og har Tillid til ; og vidne da blot hans Samvittighet » for ham , at det ved Herrens Naade har lyttes ham at vinde Wre hos sin Gud , da agter han det kun ringe , hvad enten Menneskene ere ham eller itte ; medens han dog med Glede modtager og benytter ogsaa denne Wre , forfaavidt den kan vindes og bevares uden Opossrelse af hin . Vor Herre og vor Gud , du blendes ei ved ydre Glands , og lader dig ei stusse ved udvortes Skin ; men gransker Hjerter og Nyrer og ledemmer retferdig efter Hjertets indre Vesen , o giv os , Herre , et ydmygt <b>Hjerte</b> og en reen Aand , at du kan ere og elste os som dine Born , da vinde vi den bedste , den sande Wre ; ak Herre , der er saameget ,

<h4>761</h4>faa ti. Vil du tale om det Onde , du veed eller horer om ham , da tal til ham <b>selv</b> derom med <b>Kjerlighed</b> og under fire <b>Oine</b> , og herer han dig itte , da felg Pauli Formaning , tag Een elle To med dig , paa det at Sagen maa blive fast efter to eller tre Vidnes Mnnd . Men vil dette endnu ikke hjelpe , da foredrag Sagen for Menigheden og enten den geistlige eller verdslige Ovrighed ; thi her staaer du itte alene , men med dine Vidne med dig til at overbevise dcv Skyldige , saaledes kan du paa en retferdig og ordentlig Vis hindre det Onde og bringe Forbryteren paa bedre Tanter , og du viser da ved bin Ferd , at det er <b>Kjerlighed</b> , der leder dig og christeligt Hensyn til din Nestes eget Vel ; men berer du hans Skam hemmeligt til Skue , da viser du Skadefryd og et uherligt <b>Hjerte</b> og krenker det Bde Bud . Som de alminteligste Aarfager til Krenkelse af dette Buds Fordringer , det vere sig nu ved falst elle uherligt Vidnesbyrd , nevne vi : 1 ) Sladoersyge og Enakkelyft , hvem det blot er om at gjore at have Noget at snakke om , at lade Tungen lsbe og saaledes lyver , bagtaler , hykler o . s . v. uden <b>selv</b> at vide det . 2 ) Letfindighed og Ukjerlighed, der itte tenker paa eller bekymre sig om hvilken überegnelig stor Stade han ved sin Ferd bereder Nesten ; hvor sergeligt vidner itte det daglige Liv i denne Henseende ! hvor mangen Splid og Uenighed mellem Venner og Veslegtede, hvormegen Sorg og Bedrevelse , ja Jammer og Elendighet, har itte Grund i letsindige og « herlige Rygtefmedes Sladder og Snak ? 3 ) Egen Fare , forn man derved vil afvende , egne Brest , som man paa den Maade haaber at tildette , idet man mene at formindste sine egne Feil ved at fremheve Andres , som om man <b>selv</b> blev mindre fordommelig fordi Andre itte ere bedre end vi . Hat » , Fiendfkab og Forfolgelfesfyge , for hvilke intet Middel er for lavt og nverdigt , nåar de blot kunne naae deres onde Hensigt , f . Ex . : Pharifeerne mod Frelfeen . 5 ) Menneftefrygt , idet man af Hensyn til Mennester , foråt vinde eller bevare deres Gunst fortier eller endog fordreier <b>Sandheden</b> ; endelig 6 ) Egennytte , Vindesyge og Gjerrighed , for hvem <b>selv</b> Legn og falst Vidnesbyrd er et itte forkasteligt Middel , nnnr blot egen Fordel berved knn fremmes . Det vifer sig faaledes her forn ved alle Budeue , at den sidste og egentlige

<h4>783</h4>klarere Lys , hvor de kun ufuldkomment hendes eller paaagtes, er han dog paa den anden Side heller itte blind for hans Feil , men lader disse , overalt hvor det tan stee , kun vere en Sag mellem den Feilende og ham <b>selv</b> for fortrolig Meddelelfe under fire <b>Oine</b> ; paa denne Maade viser han , at han mener Nesten det i <b>Sandhed</b> godt , og at han besjeles af den Brodeherlighedens Annd , som skildres l Cor . 13 , 4 — 7. Den Ferdighet , med Lethed nt opdnge Andres Feil er let erhvevet , men vise et hnnrdt , et uhrrligt <b>Hjerte</b> , Ferdigheden nt erkjende og bekjente deres gode Sider er vanskeligere at naae , men e en Frugt af et gjenfedt fornyet Sind . Den sidste Fordring , der efter Luthers Ord gjeres til os i dette Bud , er nt vi stulle tnge Alt i den bedste Mening . Cnlvin siger derom i sin Cntechismus : „ Det ottende Vud fordrer nf os , nt vi itte stulle vere tilbsielige til nt tenke , tale eller here det Onde , men meget mere det Gode om vor Neste , og faavidt <b>Sandheden</b> tillader det , beskytte og redde hnns gode Navn og Rygte . " Nu knu det jo desverre vistnok ikte negtes , nt det nlifor ofte er Tilfeltet, at Nestens Ferd og hans Forhold i Livet er saadan, at vi <b>selv</b> med den bedste Villie itte kan tenke Andet end slet derom , og det er heller itte her Meningen , at vi stal opt ^ ge aabenbare Ondstabssynde og Forherdelsens onde Frugter i en god Mening og tente og demme godt derom . Hvor Synden er offentlig og aabenbar , der maa og vor Tale og Dom derom vere misbilligende og dadlende , skjent altid med Idmyghet » og <b>Kjerlighed</b> til <b>Sandheden</b> og til Nesten ; men . der gives paa den anden Side ogsaa mange Mennester her i Livet , som sege en serdeles Glede og Fornsielse eller dog vist en serdeles Forherlighet ) for at fee Alt hos Andre i det msrkeste og ufordelngtigste Lys , og det ikke alene ved Nestens aabenbare Feiltrin , men <b>selv</b> der , hvor det Feilagtige ved deres Forhold knn vere tvivlsomt , jn <b>selv</b> ved det , som synes nfgjort nt vere hederligt og rosverdigt hos Nesten , forn stedse have et „ Men " , en Indvending paa rede Haand , og itte hvile fsrend de have sundet Noget at dadle , Noget at misbillige ved hans Forhold . Saadanne Mennester bere deres Christennavn med Urette , de vise at Christi Aand 3 : <b>Kjerlighedens</b> Aand itte boer i deres Hjerter , de glemme , at der er strevet : „ Med den Dom ,

<h4>833</h4>fee hans Velferd fremmes , bedroves over hans Tab og Skade , og for Herrens Skyld af <b>sand</b> Broderherlighed gjore hvad vi formnae , for nt hjelpe til nt hnn knn itte blot beholde og nyde hvad Herren har tildelt ham , men endog om mnligt sec det voxe og formeres . Men stal dette kunne stee , stal det blive os muligt endog blot med Oprigtighed at strebe til at fyldestgjere denne Fordring , da er det ogsaa en nodvendig Betingelse , at vi have et saadant Heredemme over den i vort Vesen nedlagte Begjerevne , at den itte gaacr i det blot sandselig Behngeliges , det verdslig Nyttiges Tjeneste , men med <b>sand</b> christelig Frihet » higer mod det Guddommelige , det Gud Velbehagelige , san nt vi i Alt hvad vi tegjere ferst og fremst have hans Villies Velbehngelighed for <b>Oie</b> , og vort <b>Hjertes</b> Lyst altid stane til Herren ; Pf . 37 , 4. Det Menneste , som ved Herrens Nnnde har vundet saadant Heredsmme over sit eget <b>Hjerte</b> , har , forn vi i det Foregårende skildrede , nnnet frem til den sande christelige Frihed , itte lengere trellende under Begjerligheternes Ang , bestemmer hnn med Frihed over sin Villies ttringer og den senlige Udknnrelses Aand , som har faaet Rum i hans <b>Hjerte</b> , den Kjerlighcdens Aand , der har gjort ham til et frit Guds Barn , vil vife sin Frugt i Idmyghed , Sugtmodighed , Godhed og <b>Kjerlighed</b> i ulle hans Forhold , al hans Ferd mod Nesten . Den borgerlige Lovs dede Bogstav er itte lengere Regelen for hans Opferfel mod Nesten, men den Broderherlighed , som fylder hans Sjel , hun sverger itte , hur jeg Ret til at tilegne mig dette eller hint af Nestens Gods ? men , ber jeg her benytte min Ret , kan jeg benytte den uden at stade mill Neste mere end jeg gavner mig <b>selv</b> ? Han tenker itte paa , om hnn kan forfvare for Menneskene at tilfredsstille dette eller hint Onste med Henfyn til hvad Nesten kalde Sit ; men derimot » , kan jeg elste denne Broder og dog gaae frem paa denne eller hin Muute mod hum ? vilde jeg enste ut Andre handlede saaledes mod mig ? Kort sagt , he befales os at Broderherlighedens Aand stal gjore sig gjeldende i alle vore Forhold mod Nesten , saa vi itte blot ste hen til vort eget , men ogsaa til Andres Bedste , at derfor Aanden stal heste over Kjedet i os og vise sin Frugt i vor hele Vandel ; men slantens Frugt er <b>Kjerlighed</b> , Glede , Fred ( Gall . 5 , 22 ) . Vi ville endnu til Slutning i en Hovedsum sammenfatte disfe

<h4>841</h4>Eftelevelse nf de i Budene nevnte Gjerninger , faaledes forn lederne troede , da maatte man jo dog kunne ovetrede eet Bud uden derved tillige at krenke de evrige , f . Ex . vere en Tyv uden tillige at vere en Morder , eller omvendt ; men nu da hele Loven faavelsom dens enkelte Bud flyte famnen i den ene Fordring , Reenhed , Hellighet » , faa folger naturligviis deraf igjen , at den hele Lovs Fordring , krenkes , strax eet , hvilketsomhelst af Budene , overtredes. Heraf fremgaaer da igjen , at nåar den Christne vil preve sig <b>selv</b> efter Guds Lov , fee han itte ferst og fornemmelig hen til hvad han udvortes har veret i Livet , hvad hans Mund har talet , hans Hender udfort , men gaae tilbage til Kilden , hvorfra Ordene og Gjerningene have deres Udspring , til Hjertet og undeseger dettes Tilstand , om demde findes den Reenhed og Hellighet » , forn er Betingelsen for at dets Udstrsmninger i Livet i Ord og Gjerning tan bere det Nenhetens og Hellighedens Stempel , uden hvilket de aldrig kan bestane for hnm , der grauste Hjerter og Nyrer ; thi hvad Hjertet er fuldt nf , leler Munten over med , og nf et slet Tre knu der itte heftes god Frugt ; boe der derfor ond Lyst , boer der urene Tanke og Begjeringer i Hjertets Dyb , dn vil disse nedvendig ogfnn vife sin Frugt i utjerlig Tale og syndige Gjerninger , og <b>selv</b> de for Menneskene gode Gjerninger tale alt deres Verd i Hjetehendeens <b>Oine</b> . Thomas u . Kempis siger : „ Der er to Vinger paa hvilke Mennestet kan heve sig over det Jordiske : Uskyld og Reenhed . Uskyld i Hensigt , Reenhed i <b>Kjerlighed</b> eller Tilbeielighed . Uskyld seger Gud , Reenhed gr ib er og nyder hnm . Fri i dit Indre er du kun dn , nnnr du intet andet seger og har i dit Sind , end at behage Gud og gavne din Naste . Var du indvortes god og reen , da vilde du let indsee og forstaae Alt . Er der nogensteds en <b>sand</b> Glade i Verden , saa er den visselig intet andet Sted at finte , end i et reent <b>Hjerte</b> " . Den Christne , der af et oprigtigt <b>Hjerte</b> fatte det Forfatt, redelig at strabe hvad han formaner for nt gjere Guds Fordringer i hans Lov Fyltest , vil da efter dette ogfaa indfee, at han aldrig vil kunne naae dette Maal ved at tåge for sig Bud for Vud i Loven og strebe , var det nu end nok saa redeligt , for lidt efter lidt at tilkjempe sig Ferdighet» i udvortes i Livet at efterleve fan eet fan et andet af

<h4>921</h4>dem , deels gjennem Foreldrenes gode Exempel og christelige gudfrygtige Opdrngelse og Veiledning , deels pnn andre umiddelbare Maade en serdeles Hjelp og Kraft , til ligesom Foreldrene at vandre paa Herrens Veie , og saaledes fremdeles at nyde Velsignelsen af hans Naade og atter igjen at tunne forplante den over paa sine Bern ; medens derimot ) den , som trods denne Herrens Velsignelfe og den deraf flytende Kraft og Naade , forherder sit <b>Hjerte</b> og viger af fra hans Veie , derved forer Forbandelsen over sig , og det saameget mere , jo flere og kraftigere Opfordringer , jo stsrre Hjelp fra Gud og Mennester han har havt til at gjore bedre Valg . Heri ligger jo netop en ferteles kraftig Opfordring til os Alle , til ret med Aarvaagenhet » pnn os <b>selv</b> nt agte paa Herrens Vud , af ganste <b>Hjerte</b> at elste dem og stadig have dem for <b>Oie</b> , ja uden Aftadelse at strebe til at tunue danne vort hele Liv efter dem , for derved at fere Velsignelfe og Herrens Naade itte blot over os <b>selv</b> , men <b>selv</b> over vore Bern og Eftcrkommere i tusinde Led . Ved det evige Liv forstaaes de Frelstes fuldkomne og inderlige Samfund med Gud og alle de falige Aander og al den Lytte og Salighet » , forn maa vere uadstillelig fra dette Samfund , hvor man befriet fra al den Skrebelighed og Indstrenkethed , forn heneden kleber ved vort Vefen , stal stue <b>Sandheden</b> , Gud <b>selv</b> , Ansigt til Ansigt , og der sinte Lesningen paa alle Livets Gander , Tilfredsstillelsen nf alle Sjelens Savn og Lengste og i en evigvarende Fred og Glede med de hellige Engle love Gud . Al denne Velsignelfe, denne Mistundhed tilsiger Herren Alle dem , som elste ham og holde hans Bud ; det bliver derfor nsdvendigt for os at vide , hvad det er at holde hans Bud . I de indledende Bemerkniuge , forn forudstittedes Gjennemgaaelsen af de enkelte Bud , blev der gjort opmerksom paa ( Pag . 9 — 12 ) , at Loven itte kan tilfredsstilles ved udvortes Gjerninger , men at han krever det ganste Menneste med Legeme og Sjel , Luc . 10 , 27. , eller at den hele Lov med alle dens Fordringer sinder sin Fyldestgjorelse i dette Ene , <b>Kjerlighed</b> , „ <b>Kjerlighed</b> af et rent <b>Hjerte</b> , af en god Samvittighet ) og af en ustrsmtet Tro " ( 1 Tim . 1 , 5. ) , med eet Ord , altfaa et re ent <b>Hjerte</b> . Guds Bud kan altsaa alene holdes ved et Gud velbehageligt Vesen , der har sin Rod r et reent <b>Hjerte</b> , som udvortes yttrer sig i

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112201009'target=_>Prytz, A.F.W.J., 1880, Livsbilleder fra Kristi Kirkes Historie</a><h4>155</h4>i Stilhed og takkede Gud , fordi han haude stjcenket ham en saadan Son . Det fortcelles ogsaa , at han , naar den lille Origenes sov , med andagtsfuld Taknemmelighed har kysset den slumrende Dreng , t hvem den Helligaand havde taget Bolig . Haand i Haand med Hjertets Dannelse gik nu ogsaa Aandens Uddannelse , for hvilken Faderen omhyggelig drog Omsorg . Vi erfare intet mere Bestemt om hans Omvendelse ; men saa meget er ialfald sikkert , at han allerede tidlig elskede hjertelig den Herre Jesus , og som ganske Ung begjcerede han at leve og ds for sin kjcere Frelser . Det blev ham ikke forundt synderlig lcenge at staa under sin fromme Faders velsignede Indflydelse . Aaret 203 , i hvilket Septimus Severus forbsd Overgang til Kristendommen , var Signalet til blodig Forfslgelse , iscrr i Mgypten og det svrige Nord-Afrika . Forfslgelsen rasede fornemmelig i Aleksandrien. Origenes , der da stod i sin aandelige og legemlige Ungdoms Blomst , ylsdede efter at lide Martyrdsden , og da hans Fader blev greben og kastet t Fcengsel , havde han utvivlsomt fulgt sit <b>Hjertes</b> Trang og angivet sig <b>selv</b> for de hedenske Embedsmcend , hvis ikke hans <b>Moder</b> for at undgaa dette havde gjemt hans Klceder . Anderledes dsmte han i de senere , visere Aar om dette ungdommelige Svcermeri , naar han siger :  En Prsvelse , som lroeffer os uden Foranledning fra vor Side , maa vi udstaa med Frimodighed og Fortrsstning; derimod er det dumdristigt ikke at undfly den , naar vi kunne det " . Da nu Martyrkronen blev ham felv forholdt , saa anvendte han saa meget stsrre Kraft paa af fuldt <b>Hjerte</b> skriftlig at formane fin Fader til ikke at tåge noget Hensyn til sin Familie , idet han raaber til ham :  Pas dig vel , at du ikke for vor Skyld fatter et andet Sind ! " — Leonidas var ogsaa tro indtil Dsden og beseglede sin Bekjendelse med sit Blod . Hermed begyndte for den fyrige , ligesaa videlystne som hjertefromme Ingling en tung Tid , men ogsaa en Tid , i hvis Smertesdigel hans Aands Guld og hans <b>Kjcerlighed</b> til Kristus skulde renses . Staten konfiskerede Faderens efterladte Formue , og hans <b>Moder</b> kom derved med sine syv Born , hvoraf Origenes var den VEldste , i den stsrste Nsd . Derved , at han <b>selv</b> blev optagen i Huset hos en rig kristelig Frue , blev det ham imidlertid lettere ved Afskrivning af gamle Forfatteres Vcerker at understotte Sine . Ogsaa her fik han Anledning til at bevise sig som  Manden af Staal " . En hsit begavet , af hans Velynderske i Barns Sted antagen antiokensk Lcerd ved Navn Paulus holdt i hendes Hus Foredrag af saadan Art , at han derved rokkede ved Frelsens Grundsandheder , idet han satte sine egne , verdslige , menneskevise Tanker istedenfor disse . Skarer af Tilhsrere , nysgjerrige efter det Nye , strsmmede til ; men Origenes vcegrede sig , uagtet han var Paulus ' s daglige Omgangsven , ikke alene ved at hsre paa hans Foredrag , men ogsaa ved at bede sammen med ham . Hans Afhcengighed af sin Velgjsrerinde trykkede ham ; men ved sin lernflid blev han snart istand til at lcere fra sig og kunde da skaffe baade sig og sine Underhold . Den Stund var nu ogsaa kommen , da Herren vilde give ham et stort Arbeidsfelt i sin Vingaard . Hans udmcerkede Aandsgaver , grundige Uddannelse og endnu mere hans med en streng , scedelig Vandel forbundne Troesliv gjorde ham snart saa bekjendt , at ikke faa Hedninger , der vare grebne af Evangeliets <b>Sandhed</b> , lode sig undervise af ham om Salighedens Vei . Derved, at disse kom til en grundig Omvendelse , paadrog han sig nu vistnok paa den ene Side andre Hedningers Had , og dette blev end yderligere forsget derved , at han bessgte Martyrerne i deres Fcrngsler , ledsagede dem til Retterftedet , ja talte trsstende , opmuntrende Ord til dem lige til deres sidste Stund . Det er derfor ikke at forundre sig over , at han ofte <b>selv</b> kom i Livsfare.

<h4>157</h4>Saaledes fortcrller Kirkefaderen Epifanius os blandt Andet , at man engang endog havde grebet ham med Magt , ifsrt ham en af Afguden Serapis's Presters Dragt og befalede ham Paa hedenst V ' is at uddele Palmer , og at han da uden Frygt havde sagt :  Tager Palmerne , ikke Afgudernes , men Kristi Palmer ! " Ofte omringede Soldaterne hans Hus , faa at han maatte flygte fra det ene Hus til det andet . Men paa den anden Side blev ogfaa den kristne Menigheds Opmcerkfomhed mere og mere rettet paa ham ; ifcer fandt Biskop Demetrius ^ ham stikket for en Katekets Gjerning , et Embede , forn man i Aleksandrien kun pleiede at overlade til en fcerlig Begavet . Og naar vi betcenke , at en Kateket i de Dage havde at gjsre ikke alene med uverdige Katekumener og Konfirmander , men ogfaa med dannede Hedninger , Vismamd og Lcrrde , forn snskede at blive ved Daaben optagne i den kristne Menigheds Skjsd , — at der i den Tid fandtes falfke Lcerdomme , hvis Surdeig ogfaa greb de Dannede , at felv kristelige Inglinge og Mcend snstede at komme til en videnstabelig Erkjendelfe af Kristendommen , og endelig , at Aleksandrien just var Midtpunktet for faavel Orientens forn Okcidentens kristelige Liv , faaat her altsaa blev krcevet en stsrre Dannelsens Grundighet ) end andensteds , — saa kunne vi vel bearibe , hvilket Indtryk den attenaarige Origenes maatte have gjort paa Demetrius , naar denne holdt ham stikket til at blive en Pantomus's og en Klemens ' s Efterfslger . Katekumenundervisningen foregik i Kateketens Bolig , og Mcrnd og <b>Kvinder</b> strsmmede den ganske Dag didhen for at hsre paa samme ; ja undertiden anvendte man ogfaa Nattetimerne til Undervisningen , naar Dagen ikke strak til . Undervisningsmetoden var rent kateketist : man ssgte ved Spsrgsmaalene ligesom at lokke <b>Sandheden</b> frem . Hvad det Timelige angaar , var denne Forandring i Stilling ikke nogen Fordel i den fsrste Tid . Han maatte selvfslgelig indsircenke sin private Undervisning. Den Godtgjsrelse , en Katekumen nu og da kunde yde sin Lcerer , var ikke af stor Betydning , og Origenes maatte for at erncere sig og Sine kjsbe kostbare Haandflrifter af gamle Vcerker , forn han da felv affkrev og solgte , og paa den Maade kunde han bringe det til at tjene det Allernsdvendigste. Han maatte ikke alene leve overordentlig tarvelig ; men han tog Aar efter Aar faa liden Fsde til fig , at hans Helbred led derunder . Han arbeidede faagodtsom den hele Nat , og naar han fov , var den haarde Jord hans Leie . Naar vi faa ogfaa vide , at Origenes bogstavelig opfyldte paa sig felv Herrens Ord i Matth . 19. 12 , idet han lemlestede sig <b>selv</b> , forat han derved som den , der ogsaa havde at give Koner og Piger Undervisning , kunde fri sig fra alle stygge Rygter , saa ere vi ikke bange for at sige , at han oversaa , at den onde Lyst har sit Seede i Menneskets <b>Hjerte</b> , og at Verdens Skik til at tale allehaande Ondt om Guds Bsrn ikke lader sig beseire. Vi finde heri kun et Bevis for , at han endnu ikke havde naaet den aandelige Modenhed , der fatte ham istand til at stille det Sygelige fra det Sunde og egne Gjerninger fra Guds Bsrns Frihed i Kristus Jesus . Men vi tro dog at vere forvisfede om , at dette Svermeri ikke kom afLyst til ved egne Gjerninger at oprette en egen Retferdighed , men af et i den inderligste <b>Kjerlighed</b> til Frelferen begrundet fcedeligt Alvor . Herren fsrgede ogfaa for , at Origenes i Livets tunge Korsskole blev belcert om her at treffe det Rette mellem paa den ene Side en Kjsdets udvortes Spcegelfe , der kunde friste Hjertet til falsk Indbildning om egen Dygtighed til at bekjcempe det Onde , og paa den anden Side en Slaphed og Frihed , der er blottet for alt virkelig kristeligt Alvor .

<h4>245</h4>Et smukt Udtryk for , hvad der raadede i Vnglingens <b>Hjerte</b> , have vi fra den Tid , da han levede sammen med sin <b>Moder</b> og Bedstemoder i Pontus ' s enfomme Egne .  Vi maa fremfor Alt " , saa strider han ,  strcebe efter Ro for Aanden ; ellers kan Viet ikke tydelig erkjende <b>Sandheden</b> . Hvad er faligere end her paa Jorden at efterligne Englenes Kor , end straks ved Dagens Frembrud at hceve sit <b>Hjerte</b> i Bsn og prise Skaberen med Lovsange og aandelige Viser og saa ved Solens klare Lys at gaa til Arbeidet , overalt under Bsn , idet man krydrer sin Gjerning med Lovsang som med Salt . Den stille Ensomhed er Begyndelsen til Sjcelens Renselse " . Basmus fik dog fole , at Ensomheden bod ham mmdre , end han havde haabet .  leg har vel forladt Opholdet i Byen som en Kilde til utallige Onder ; men jeg har dog ikke kunnet efterlade mig <b>selv</b> . Idet jeg ligesom I maa bcere paa de i mig boende Lidenskaber , bliver jeg overalt kvalt af den famme Uro , saaat jeg ikke blev synderlig hjulpen af denne Ensomhed " . At en Mand af Basilius ' s udmcerkede Fromhed og grundige Uddannelse maatte henlede Opmcerksomheden paa sm , er saa rimeligt . Han blev ogsaa i Aaret 364 af Biskop Eusebius af Ccesarea udncevnt til Presbyter i denne Om hans Virken i det hellige Prcrdikeembede kan fslgende Brudstykke af en Tale , som han engang holdt efter en mislykket Hsst , tjene som Prove .  Lader os nu som fornuftige Mennesker efterforste Grunden til denne Uorden i Naturen . Er han ikke mere , der behersker den hele Verden ? Har Gud alemt sin Husholdning ? Er han bleven bersvet Kraft og Magt ? Eller har Gud forandret sin Godhed og Forsorg mod os til Mennestehad ? Intet fornuftigt Menneske vil sige det . Men klar og indlysende er Grunden til , at vi ikke blive regjerede som ellers . Vi modtage ; men vi dele ikke med Andre . Vi takke for Velgjerninger ; men vi bersve Trcengende dem . — — — Vi forbarme os ikke over vore Medtjenere . Vi have Hunger og blive mcrttede; men vi gaa forbi den Trcrngende . Vi have i Gud en rundhaandet Giver og Skatmester ; men vi ere gjerrige og dele ikke med den Fattige . Derfor truer den strenge Dom os . Gud aabner ikke sin Haand , fordi vi have lukket vore Hjerter ' for Broderkjcerligheden . Derfor ligge vore Marker tsrre , fordi <b>Kjcerligheden</b> er bleven kold . Du Syndbefmittede ! trcrd frem , kncel ned , grced og suk ! Ssnderriv de uretfcerdige Gjeldsbeviser , tilintetgjsr Aagerkontrakterne ! Men denne Tid er ogsaa til Provelse . Nu viser det sig , om den Rige er gavmild og elsker Brodrene , — om den Fattige er taknemmelig og ikke henfalden til Laster . Laan den rige Gud paa Rente ! Kan <b>selv</b> bespiser dagligen Mange " . I Aaret 370 blev han Bistop af Ccrsarea . Han skulde have Opsynet med den kirkelige Forvaltning i halve Lilleasien , og han tog det scrrdeles alvorlig hermed . Taalmodig , venlig og eftergivende , hvor han kunde , var han übsnhsrlig , naar det gjaldt at opretholde Orden og Tugt i Kirken . Befalende, advarende , opmuntrende , trostende , snart paa et Sted , snart paa et andet , snart ligeoverfor Menighederne , snart ligeoverfor den Enkelte , baade Prester oa Lcrafolk , baade Mcend og <b>Kvinder</b> , ssgte han i Alt at vcere et Monster for sin store Hjord . Uagtet han havde betydelige Indtcegter , var han fattig ; han eiede for hvergang kun een Klcedning . Brod og Salt med Vand var hans sedvanlige , haardt Brsd og Kaal med lidt simpel Vm hans bedste Fsde .

<h4>325</h4>efterham , svarede han :  lohannes — naar blot ikke de Kristne havde Malet ham ! " Efter at have tilendebragt sin videnfkabelige Uddannelse nedsatte Krysostomus sig som Sagfsrer , fordi han troede paa dette Felt bedst at kunne udnytte sine Gaver . Men de mange Renker og Kneb huede ham ikke ; han lengedes efter en Gjerning , der kunde tilfredsstille hans higende <b>Hjerte</b> , og han fslte , at det offentlige Livs Fordervelse fjernede ham mere og mere fra det , der dog var og blev hans <b>Hjertes</b> sodeste Opfriskelse . Paa denne Tid Mede Krysostomus Bekjendtstab med sin Fedrestads erverdige Biflop , Meletius . Denne indtog under de vilde Partikampe mellem Rettroende og Arianere en forsonligere Stilling , uden at han dog <b>selv</b> afveg fra den rette Lere . Men som han saa , at der Paa begge Sider var megen Bitterhed , og at der blev kjcempet mere med Kjsdets end med Aandens Vaaben , fandt han , at han burde forssae at vere <b>Sandheden</b> tro i Kjerlighed. Dog — som det stedse gaar — han hsstedekun Utak , Mistro , ja Had fra begge Sider . Han forvaltede Bispeembedet i det Hele over tyve Aar med Trofkab og evangelisk Mildhed . Som det stede saa ofte paa den Tid , maatte ogsaa han i sin Herres Tjeneste cede Landsforvisningens bitre Brsd . I tre Aar nsd Johannes Krysostomus denne cedle Mands Omgang og Undervisning . Ved Enden af denne Tid blev han dsbt og beseglede derved det forlengst fattede Forfcet helt og holdent at vie sig til Herrens Tjeneste . Helst havde han nu brudt med Verden og hengivet sig til et stille Eneboerliv. Men Meletius ' s og Moderens Indflydelse holdt ham tilbage derfra . Saa begyndte han sin krkelige Bane som Lektor . Som Saadan erholdt han en Art Ordination og maatte i Forsamlingerne forelcrse de hellige Bsger ; men han hsrte ikke egentlig med til Geistligheden , der forrettede ved Alteret . Da der til Lektorstillingen ikke krcevedes synderlig Forbannelse , brugtes undertiden unge Gutter dertil . Saaledes havde Keiser lulian i sin Ungdom maattet vcere Foreleser . Under denne sin fsrste Virksomhet » i Kirkens Tjeneste kunde han ikke mere glede sig ved at nyde sin faderlige Ven Meletius ' s Veiledning . Thi Keiser Valens havde i Aaret 370 sendt ham i Landflygtighed . Hans Post blev imidlertid rsgtet af Presbyterne Evagrius og Diodor . Denne Sidste , der senere blev Biskop af Tarfus og en meget bersmt Mand , var Banebryderen for en ny Retning inden Theologien , der scerlig lagde Vegt paaSkriftfortolkningen og holdt den sunde Middelvei mellem paa den ene Side en lss sindbilledlig ( allegorisk ) og en tsr Bogstavfortolkning paa den anden . Denne Retnings Opgave var ved Efterforskning af den hellige Skrifts enfoldige , ligefremme Betydning at grundlcegge en cegte bibelsk Theologi . Og hos Krysostomus merker man tydelige Spor til , at han er bleven paavirket fra denne Kant . Vi maa her nevne en anden fremragende Difcipel af Diodor , nemlig Theodor , fenere Biflop i Mopsuestia i Cilicien . Disse to unge Mend bleve knyttede til hinanden i et inderligt Venskab og have begge stor Betydning for Kirken . Deres Lerer Diodors Eiendommeligheder udviklede sig hos dem paa noget forskjellig Maade : Theodor slog ind paa en mere videnstabelig , Johannes derimod paa en mere praktisk-kristelig — saa at sige — pastoral Vei . Forgjeves anstrengte Krysostomus ' s Ungdomsven , Basilius , der allerede forlengst havde stuttet sig til Munkene , for at faa ham til at komme til sig . Den barnlige <b>Kjerlighed</b> holdt ham tilbage . Han levede stille og tilbagetrukken, medens hans <b>Moder</b> omhyggelig Mgede for alle hans Behov og strengt drog Omsorg for , at han fik vere uforstyrret af den hele Udenverden ,

<h4>372</h4>hendes Mne , for han havde ophsrt med sit syndige , Liv . Hendes arme <b>Hjerte</b> bar dog med Vanskelighet ) den af hende <b>selv</b> paalagte Byrde . Da trsstede Herren hende i en Drsm . Det syntes hende , at hun stod paa en smal Sti , og en Ingling af glimrende Skikkelse traadte frem for hende og spurgte , hvorfor hun var saa bedrsvet . Hun klagede over sin ståkkels, i Syndens Elendighed liggende Ssn . Men Engelen talte til hende og bad hende se op ; thi der , hvor hun stod , der stod Ssnnen ogsaa . Og da hun saa op , stod ogsaa Ssnnen ved hendes Side . Saasnart hun vaagnede , ilede hun glad afsted for at meddele Augustin dette Drsmmesyn . Men han udtyder Drsmmen derhen , at ogsaa hun skal gaa over til Manikceismen . Monika svarede imidlertid raskt :  Der blev ikke sagt : hvor han staar , staar ogsaa du ; men : hvor du staar , staar ogsaa han " . Augustin erkjender , at den Aandsncervcerelse , hvormed hun sagde dette , traf ham mere end Drsmmen <b>selv</b> . Imidlertid rakte hun nu Ssnnen Haanden til Forsoning , modtog ham og hans Ssn i sit Hns og ssgte at paavirke ham til det Gode . Ogsaa en anden Trsst blev Monika tildel . Hun henvendte sig til en Biskop , der <b>selv</b> engang havde vcrret Manikceer , med Bsn om , at han maatte bringe hendes Ssn bort fra denne Vranglcrre . Men han undslog sig .  Lad ham vcere i sin Nyfigenhed , at han ikke skal blive forstokket . Hans Time er endnn ikke kommen . Bed kun for ham til Herren , saa vil han nok <b>selv</b> ved Lcesning og Forsken finde , hvor stor og gruelig en Vildfarelse han er hildet i " . Men da hun fremdeles troengte ind paa ham , at han skulde tale Ssnnen tilrette, sagde han halvt uvillig :  Gaa kun ! Saasandt du lever , det er ikke muligt , at en saadan Taaressn kan gaa fortabt " . — Dette greb hende , som om det havde vceret en Rsst fra Himlen . Hun vedblev i Troens Frimodighed at bede om Ssnnens Omvendelse . Augustin blev nu truffen af en svcer Hjemssgelse . Han fortceller <b>selv</b> dette saaledes :  I hin Tid , da jeg havde begyndt Lcerervirksomheden i min Fcedrestad , havde jeg erhvervet mig en Ven , der formedelst Ligheden i Studier var mig meget dyrebar , og som ligesom jeg var i sine bedste Aar . Han var opvoksen som Gut sammen med mig ; vi havde bessgt den samme Skole og legt med hinandm . Desuagtet var dette Forhold ikke et saadant , som maa finde Sted i det sande Venskab . Thi der gives ikke noget andet <b>sandt</b> Venskab end det , som du , o Gud , knytter mellem dem , som hcenge fast ved dig i den <b>Kjcrrlighed</b> , som er udgydt i vore Hjerter ved den Helligaand , der er os given . Dog var dette Venskab meget ssdt , da det ftemglk af Begeistring for de samme Studier . Jeg havde fsrt ham bort fra den sande Tro , af hvilken han som Yngling var dybt og helt gjennemtrcengt , til hine overtroiske og skadelige Fabler , for hvis Skyld min <b>Moder</b> begrced mig . Han for med mig om paa vildfarende Veie , urolig i Aanden , og min Sjcel kunde ikke leve noen ham . Og se , du forfslger dem , der undfly dig , du Gud , — — og omvender os til dig paa nnderlige Veie . Se , du tog Vennen bort fra dette Liv , da dette Venskab , der var mig ssdere end alle mit daværende Livs Glceder , havde bestaaet knapt et Aar . Hvem kan tcrlle alle dine store Gjerninger , som een Eneste har erfaret i sit Liv ? Han laa i Feber bevidstlss, i Dsdssved . Da man opgav alt Haab , blev han dsbt , altsaa i sin bevidstlose Tilstand , uden at jeg dog bekymrede mig derom , og idet jeg forudsatte, at hans Sjcel fsr vilde beholde , hvad jeg heldte i ham , end hvad der uden hans Vilje stede med hans Legeme . Men det stede anderledes . Han blev sund . Saasnart jeg kunde tale med ham — hvad jeg gjorde , saasnart han sormaaede det , fordi jeg ikke veg fra ham , og vi vare nadfkillelige — forssgte jeg at faa ham til med mig at spotte over den Daab , som han havde

<h4>376</h4>Efterat han var bleven frist igjen , henvendte den romerske Prefekt Symmakus i Mailand sig til ham med Anmodning om at sende ham en duelig Lcerer i Veltalenhedskunsten . Augustin <b>selv</b> fulgte Kaldet og drog til denne By . Sammen med ham fulgte ogsaa en gammel Skolekamerat fra Tagaste , den edle Alypius . Augustin fandt sin Barndomsven igjen i Rom som romersk Embedsmand , nemlig som Skatmester for Provinsen Italien . Ved sin Übestikkelighed og Embedsdygti ^ hed erhvervede han sig almindelig Agtelse . Augu ^ stin og han bleve nu uatskillelige for Livet . Endnu en Landsmand , den hsit begavede Nebridius , var med. Nebridius havde holdt sig ren for alle det karthagiske Studenterlivs Laster og havde allerede af den Grund vundet Augustins Agtelse . Han havde forladt sin Familie og sin stjsnne Landeiendom for at fsge efter <b>Sandheden</b> . Disse tre unge Mend var det altsaa , som reiste sammen til Mailand . De hungrede efter <b>Sandheden</b> , men foragtede dog den Spise , forn alene kunde mcette dem . Til dem kom Monika — dreven af sin inderlige <b>Kjerlighed</b> til Ssnnen . Fuld af Haab reiste hun afsted over Havet , trsstede iet Uveir de forsagte Ssfolk med , at et Syn havde forjcettet dem en sikker Overfart , og modtoges ved sin Ankomst til Mailand af det glade Budfkab , at hendes Son havde sagt sig fuldftendig lss fra Manikeerne og endog efter nogen Tvivl havde meldt sig for at modtage Undervisning som Daabslerling . Dette var hende ikke nogen Overraskelse ; thi hun havde jo bedet derom med Troens fulde Frimodighed ; nu kunde hun ogsaa sige til ham med et glad <b>Hjerte</b> :  Min Ssn , jeg vil , fsrend jeg dsr , komme til at se dig som en rettroende Kristen : det tror jeg i Herren " . Guds kjcerlige Fsrelse viste sig nu deri , at han kom under Paavirkning af den fsr ( Side 65 og fag . ) omtalte Biskop Ambrosius , en af Herrens mest benaadede Tjenere . Ambrosius ' s Predikener vare i denne Tid af stsrre Betydning for ham end noget Andet . Augustin hsrte fsrst Bistoppen paa Grund af dennes store Veltalenhed ; men Predikenernes grundige , alvorlige Indhold svede snart en meget sterk Indflydelse paa hans <b>Hjerte</b> , og scerlig fengsledes han af den Maade , hvorpaa den begavede Herrens Tjener tolkede det gamle Testamente , der hidtil havde vceret Augustin en fuldstcendig lukket Bog , og satte det i levende Forbindelse med det nye . Dette Indtryk blev yderligere forsterket ved hans Omgang med Biskoppen . Men Augustin var endnu langt borte fra <b>Sandheden</b> . Han var stcerkt dragen . Han stod under Aandens Arbeide . Men der var Meget , som skulde knuses hos ham . Fsrst vilde han fatte Kristendommen ad Forstandens Vei ; men det er en Umulighed ; thi  det naturlige Menneske fatter ikke de Ting , som hsre Guds Aand til ; thi de ere ham en Daarlighed , og han kan ikke kjende dem ; thi de bedsmmes aandeligen " . Saa mente han ved egen Kraft at stulle kunne forbedre sit lastefulde Liv . Men dette kunde han felvfslgelig ikke ; det blev kun Nederlag paa Nederlag . Han fsrte en pinlig Tilverelfe .  Du min Gud " , udraaber han ,  du alene kjender , hvad jeg led , men intet Menneske " . . Moderen fattede nu den Plan , at hun ved et ordentligt JEgtefkab vilde fsre Ssnnen bort fra den fyndige Forbindelse med sin Bolerske , da han nu engang syntes uskikket for det ugifte Liv . Der fandtes ogfaa en ung , cedel Jomfru , der vilde gifte sig med ham ; Brylluppet skulde kun opsettes et Par Aar Paa Grund af hans Forlovedes unge Alder . Som en megtig Hindring for ZEgtefkabet stod dog hin <b>Kvinde</b> , med hvem han havde levet sammen i mange Aar , der havde fsdt ham Ssnnen Adeodatus , og til hvem han var bunden med mange sterke Kjsdets Baand . Han erklcerede , at han ikke kunde lade sig stille fra hende . Da rsrte Gud ved hendes <b>Hjerte</b> , og af Kjcerlighed

<h4>562</h4>Desuden fik han ogsaa kraftig fole , at den gamle Adam i ham ikke var udryddet. De strenge Gudfrygtighedsovelser , som han hidtil havde iagttaget , havde kastet en Glorie om ham som en hellig Guds Mand ; men Egenkjerligheden havde ikke havt godt as , at han saaledes blev « ret , og naar saa Gud tvang ham til at give Afkald paa sit strenge asketiske og selvplagende Liv og lod ham gaa i Idmyghedens Hoiskole for grundig at kurere ham for hans Hovmod , saa voldte dette mange bitre Stunder . Inden selve Klostret maatte han ogsaa doie meget Ondt . Man kan let tcenke sig , at en saa fin og dyb Natur som Susos ikke blev forstaaet og vurderet af de mange tildels ligetil raa Personer , som befandt sig inden et Klosters Mure . For dem var den underlige Drommer , der , som ' de sagde , ikke kunde gjsre Andet end  glo paa Himlen " , til Spot og Latter . Klostertugten var saa slåp , at han ofte sad ved Bordet som en Skive for deres flaue , raa Vittigheder , saaat han gredende trak sig tilbage til sin Celle . Ogsaa de af ham forfattede Bsger , der gik tillaans Mand og Mand imellem , vakte Anstod. Den , der manglede et dybere <b>Blik</b> , kunde let forveksle hans Udtalelser i disse Skrifter med de Lerdomme , som hyldedes af de ved Rhinen dengang temmelig udbredte saakaldte  frie Aands Brsdre " , som fuldkommen ophcevede Forskjellen mellem Gud og Verden og i sit Svcrrmeri endog gik saa vidt , ar de sagde , at der ikke fandtes nogen Synd hos dem , som dreves af Guds Aand ; Alt , hvad saadanne Mennesker gjorde , var rigtigt , <b>selv</b> om det af Lovens Bogstad blev stemplet som Synd . Det er en Selvfolge , at de derved aabnede Doren for den fceleste Kjsdslyst . Susos monsterverdige Liv beskyttede ham ikke for mangeslags hadefulde og taabelige Anklager , og om han end blev frikjendt for al Vranglere , gjorde dog den blotte Mistcenksomhed ham hjertelig ondt . Dog just i denne Mmyghedens Skole lcerte han den gyldne Kunst at taale allehaande Ondt og give Velsignelse for Forbandelse , <b>Kjerlighed</b> for Had , den Kunst , hvori vor velsignede Frelsermand staar som det hsie , herlige Forvillede . Dog endnu forestod der ham en Ildprsve af en forferdelig Art . Vi forbigik gjerne Altsammen , dersom det ikke gav os Anledning til at kaste et dybt <b>Blik</b> md i de tunge Fsrelser , hvorigjennem denne Herrens Tjener skulde modnes for Himlens ssde Hvile . Suso havde i sin vidtomfattende Sjelesorg allerede lcenge gjort sig scerlig Moie med en usedelig <b>Kvinde</b> ; men , da han havde merket , at hendes Anger og Ruelse kun var Hykleri , trak han sig tilbage fra hende . For at hevne sig derfor udgav hun ham for Fader til et Barn , som hun havde fodt . Dette var et Slag , som traf ham paa det Smerteligste ; giftige Tunger bredte Historien vidt udover , og <b>selv</b> Saadanne , der hidtil havde elsket og eret ham , troede Lognen . Den erede , bekjendte Hellige stod der da med eengang som den afdekkede , blottede Bedrager ; hele hans Orden var bleven beskjemmet. Det var den tyngste Tid i hans paa alskens Trengsler saa rige Liv . Dag og Nat drog han om fra den Ene til den Anden . Ingen vilde tåge sig af ham , og <b>selv</b> hans allerbedste Venner , hos hvem han sogte Trost , viste ham haardt og uvenlig fra sig . Tilsidst overfaldtes han af en <b>sand</b> Helvedes Angst , og den Ulykke , der havde rammet ham , laa som en centnertung Byrde paa hans beklemte <b>Hjerte</b> . En halv Dag vårede den skrekkelige Kamp ; da blev hans <b>Hjerte</b> stille uuder Bsn til Gud , og fredfuldt og mildt loddet fra det forpinte <b>Hjerte</b> :  Skal det ikke vere anderledes , Herre , saa ste din Vilje ! " Men nu havde Herren opnaaet , hvad han vilde ; og nu lod han ester Ulykkens Stormskyer Solen igjen stinne frem , — og dobbelt deiligt var dette efter de mange tunge Dage . En af hans Orden nedsat Undersogelses-

<h4>564</h4>komnnssion kunde godtgjsre hans Uskyldighet , . Flere af dem , som havde krenket ham mest , bade ham om Tilgivelse , og Gud gav nu den stakkels Korsdrager en Fylde af inderlig , dyb Hjertefred . <b>Selv</b> havde han lert saameget i denne tunge Ildprsve , at han takkede Gud derfor paa sine bare Kne som den stsrste Naadesbevisning i sit Liv , den , han ikke havde villet undvere for alle Verdens Skatte . Der staar nu kuns tilbage al kaste et <b>Blik</b> paa Henrik Susos Virken , sorsaavldt den var mere mcerkbar i det Idre . At en Mand med Susos rige aandelige Erfaring og fslsomme Sind maatte blive en ualmindelig Predikant , var at vente . Naar Suso eller , , den Ssde " , som Folket kaldte ham , stod paa Preikestolen , lyste hans Ansiat vidunderlig op , fortcelles der . Men stsrre end som Taler var han dog som Sjelesorger ; det var ogsaa det Felt , hvortil hans naturlige Anlceg iscer henviste ham . Allerede fra sin Ungdoms Dage as bar han Tilnavnet Amandus", den Elskelige ; thi som hans <b>Hjerte</b> flog varmt og kjerlighedsfuldt for Gud , saaledes omfattede han ogsaa sine Medmennesker med den inderligste Kjerkghed .  leg havde ester min <b>Moder</b> " , forteller han om sig <b>selv</b> ,  faaet et mildt <b>Hjerte</b> , og aldrig saa jeg et Menneske lidende eller bedrsvet , uden at jeg havde den hjerteligste Medlidenhet , . Alle Mennesker , som traurige eller bedrsvede kom til mig , fandtßaad hos mig ; thi jeg grced med de Gredende og ssrgede med de Bedrsvede , indtil jeg igjen bragte dem til godt Mod . <b>Selv</b> Fuglenes og Dyrenes Sorg gik mig til <b>Hjerte</b> , og jeg bad den milde Herre , at han maatte hjelpe dem . Alt , som lever paa Jorden , fandt Naade og Mildhet , for mine Dine " . Med et saa varmt og kjcerlighedsfuldt <b>Hjerte</b> maatte han , saasnart han havde overvundet sin oprindelige Skyhed , kunne virke velsignet i Sjelesorgen . Hans Virkekreds strakte sig vidt udover Klostrets Mure ; hyppig finde v : ham paa Reiser , mest nedover Rhinen til Ksln , Aachen og Neverlandene , hvor hans Ven Johan Roisbruk virkede ganske i hans Aand ; ogsaa i Zyrick skal han ester en gammel Overlevering have veret temmelig ofte — for overalt , hvor han kom , at prise den velsignede lesusmand og opflamme de kolde Hjerter til ny <b>Kjerlighed</b> . Hans Virksomhet» havde noget Ensidigt ved sig ; han trak Hjerterne bort fra <b>Kjerligheden</b> til Verden til Guds ssde <b>Kjerlighed</b> ; men i denne Ensidighed ligger netop det Trylleri og den underbare Magt , hvormed han vidste at gribe Hjerterne . Hvor han paa sine Reiser msdte et Menneske , begyndte han straks at tale til ham om den kjere , milde Gud , der havde syldt hans eget <b>Hjerte</b> med saamegen aandelig Glede , og det hendte ikke sjelden , at forstokkede Verdensbsrn , der havde holdt sig borte fra Ord og Sakrament i aarevis , bleve grebne af hans Ord og vundne for Livet i Gud . Hans aandelige Bsrn , til hvis Omvendelse han havde veret Midlet i Guds Haand , vare spredte vidt omkring ; af disse ere serlig blevne nevnte den allerede omtalte Nonne Elisabeth Steglin og en borgerlig Frue Anna . Som nu disse erede sin aandelige Fader , saaledes hang han med den inderligste <b>Kjerlighed</b> fast ved dem og bar dem ofte i brendende Bsnner frem for Naadens og Barmhjertighetens Throne . Han gjorde ogsaa lange Reiser for at se en eller anden af dem og styrke dem i Troen , og vi besidde endnu en Rekke af Breve , som han har strevet til disse ham aandelig forbundne Sjele . Serlig stor var Suso , naar det gjaldt Redningen af en Forloren . I Nonneklostrene forekom det dengang temmelig hyppig , at de yngre Nonner bleve sit Lsfte utro og gik tabte i Verdens vilde Larm under et utugtigt og verdsligt Liv . Hsrte nu Suso om et saadant Fald , saa drev det ham til milevidt over Berg og Dal , gjennem Storm og Kulde at drage efter det for-

<h4>827</h4>Og hvilke Vaaben bleve ikke ogsaa brugte mod ham ! En Ven skrev til ham :  Der ligger beredt dsdbringende Gift for at rydde dig afveien . Udenfor Hjemmet tor du aldeles ikke nyde Noget med Sikkerhet ) . " Man forssgte at kalde ham til Syge og Dsende for da hemmelig at kneble ham og fore ham bort . Man kastede ikke alene md Vinduerne i hans Hus ; men Snigmordere gik efter ham for at drebe ham . I <b>Sandhed</b> , mange Kors , megen Forfslgelse ! Men saameget rigere trsstede Herren ham ved den store <b>Kjerlighed</b> , hvormed alle hans og Evangeliets Venner hang ved ham , — ved Evangeliets feierrige Fremtrengen fra Zyrich af , men fremfor Alt ved Ordets Trost , faa hans <b>Hjerte</b> dog trods alt dette blev stille og fredfuldt i Herren .  Kristi Evangelium har den Egenstab , at det bedst viser sin Kraft og udbreder sig mest under Forfolgelfer . Kristus har udgydt sit Blod for vor Frelses Skyld , og det er en feig Krigsmand , der ikke vil give fit Blod for sin Herre ogHcerfsrer , men viger tilbage , medens hans Herre har lidt Dsden for ham . De rette Kristi Stridsmend ere beredte til at lade Hovedet knuse for sin Herre . " Disfe Zwinglis Ord vise os det Sindelag , med hvilket han gik Forfslgelfe og Lidelse imsde . Imidlertid vandt de Evangeliske Seier paa Seier . Franciskanermunken Frans Lambert disputerede i Aaret 1522 med Zwingli , men erklcerede sig overvunden og blev senere en af Evangeliets Forkjempere . Den 29 de Januar det fslgende Aar afholdtes igjen paa Foranstaltning af Statens Raad en offentlig Disputation .  Guds Aand har nsdt mig til at tale , han ved ogfaa , hvorfor han har udvalgt mig Nverdige dertil . Velan da i Guds Navn ! Her er jeg , " — det var Zwinglis Hilsen til den af over 600 Personer bestaaende Forsamling . Han havde foran sig paa et Bord den latinske Bibel , det hebraiske Gammeltestamente og grceske Nytestamente ; den hellige Skrift var det Vaaben , hvormed han vilde kjempe . De Forsamlcde bleve opfordrede til at optrede imod ham . Generalvikaren hos Bistoppen i Kostnitz , Johan Faber , forfsgte ogsaa at forsvare de papististe Lerdomme og Skikke , men forstummede aldeles , da Zwingli opfordrede ham til at bevise fine Paastande af den hellige Skrift . Ingen greb senere Ordet . Man gik fra hverandre ved Middagstid , og da man igjen forfamledes om Eftermiddagen, forelagde det hoie Raad sin Beslutning om , at Magister Ulrik Zwingli skulde fortfare med efter bedste Evne at forkynde det rette guddommelige Ord . Disputatsen var forbi , Fienden taug , Zwingli og med ham Evangeliets Sag havde vundet en glimrende Seier . Zwinglis reformatoriske Kald traadte stedse klarere frem for Dagen . Hans Prcedikener begyndte ogsaa at virke . Der kom en mere evangelist Aand md blandt Folket . Kirkegodsernes Udbytte blev anvendt til at lsnne evangeliske Predikanter med , Klostrene bleve forladte af Munke og Nonner , Klosterbygningerne bleve benyttede til Sygehuse og Skoler , og Geistlige giftede sig — saaledes ogsaa Zwingli <b>selv</b> , der i en from Enke fandt en trofast <b>Hustru</b> , der var besjelet af sin Mands Troes- og Lidelsesmod . Den 26 de Oktober 1523 vare atter mere end ni hundrede Personer forsamlede paa Raadhuset for at oververe en Disputation om Billeddyrkelsen og Messen . Zwingli sagde <b>selv</b> :  Det vere langt fra mig at forkaste Malerier og skjsnne Bllledststter , naar man ikke yder disse overtroiske Mresbevis ninger ; der er vel ikke Nogen , som ser saadanne hellere end jeg , ja , jeg erkjender, at Maler- og Billedhuggerkunsten ere Guds Gaver . Men alle Billeder stulle blive fjernede , der blive viste overtroiske æresbevisninger . " ' Disputationen om Billederne var snart endt . Den gamle Korherre , Konrad Hoffmann , der optraadte til Forsvar for Billeddyrkelsen , og forn sluttede

<h4>936</h4>stadigt Bsnneforholo til Herren , med levende Trang sugede aandelig Ncrring af Ordet og intet Bedre , intet Hsiere vidste end at fsre andre dyrekjsbte Sjcele til den Frelser , som havde forbarmet sig over ham og gjort ham saa glad . Han var Pastor med Liv og Sjcel . Paa den Mindesten , der hviler over Herbergers Grav , staar derfor ogsaa denne Indskrift :  Valerius Herberger, hvis <b>Kjcerlighed</b> var Jesus , hvis Fryd var Jesus , ja hvem Jesus var Alt , venter her paa Jesu Opstandelse . " Og dette Vidnesbyrd om ham er den fuldeste <b>Sandhed</b> . En usigelig stor <b>Kjcerlighed</b> til Jesus gjennemtrcengte og beherskede hans hele Liv baade i Smaat og Stort . I hans Dagbog Mdes der fast ved alle de Ting , som der ere anmcerkede , korte Bonnesuk paa Latin eller Tyst :  lesus , ' se til os ! — Jesus , afvend Faren ! — Jesus , vcrr vor Jesus ! " osv. Har han faaet en Gave , saa heder det :  lesus , dig verre Pris ! — Wre verre Gud i det Hsieste ! — Jesus , du verre lovet i Evighed ! " osv. Omtaler han , hvorledes Stemningen i hans Hus er bleven forandret , saa skriver han :  Herre Jesus , du hersker over alle Hjerter , led os ved din Helligaaud , at denne Forandring maa blive til vor Families Fordel ! " Under hans Bouneliv vare Salmerne ham i Besynderlighet ) kicere . Et Stipendium , som han havde faaet som Student , forpligtede ham til hver Ssndag at bede de 7 Bodssalmer . Han fsiede <b>selv</b> til det Lofte , at han daglig vilde bede Salmer , hvad han ogsaa trolig gjorde af Hjertens Lyst Morgen og Aften . Han anvendte Dagens bedste Timer til Studium af Bibelen .  Den hellige Bibel er min stjsnneste , kjcrreste Have , " sagde han . For at faa Tid til sin Lcesniug i Bibeleu modtog han sjelden nogen Indbydelse til selskabelige Sammenkomster ; det kunde hcrnde en enkelt Gang , at han brsd denne Regel for at kunue verre sammeu med en kjcer Ven eller deltage i en sceregen Festlighed . Den Aand , der raadede i hans <b>Hjerte</b> , trcengte ogsaa mere og mere md i hans Hus , Hans <b>Hustru</b> Anna , ' Datter af Raadsherren Rudiger , maa have vcrret ham en ligesindet Hjcrlperinde . Hau kalder hende <b>selv</b> med Tat til Gud en Wgtehustru fuld af Gudsfrygt og Dueenfold , en trofast Hjcelperste i Tro , i Liv , i Bsn og Sorg . Hans seksaarige lille Son Valerianus's Dsdsleie giver Vidnesbyrd om den Aand , som raadede i Huset . Barnet strakte i sin Angst sine smaa Hcrnder mod Himlen og raabte : '  O , du sode Jesus , hjcelv mig dog ! O tom dog , jeg gaar saa gjerne did op ! Hvor har du skjult dig ? Lad mig dog faa dig , hjcelp mig dog ! " Noget senere gav Barnet sig <b>selv</b> Svaret :  la , ganske vist , jeg vil forlsse dig . " Da Moderen spurgte :  Vil du ikke blive hos mig ? " svarede Barnet :  Nei , til mm Herre maa jeg . " Mange , der stode omkring det deilige Dsdsleie bevidnede , at de aldrig havde hsrt et Barn komme med saadanne Ord . Det rige Hjerteliv , som Herberger levede med sin Gud , og som scerlig styrkedes og uceredes i stille Bsnne- og Bibeltimer , det er de Rsdder , hvorfra hans ydre Liv sprang frem . Hvad han samlede i sit stille Kammer , det strsmmede i rig Fylde udover Menighedeu i hans Prcrdikener . Han var en scrrdeles begavet Prcedikant , hvad hans trykte Prcedikener noksom bevise . De ere saa kraftige og opbyggelige , at den gamle Herberger endnu den Dag idag i mange Egne er en kjendt og elstet Hnsven . ' Det er rige Skatte , som han henter frem af det Guds dyrebare Ord , hvormed han var bleven faa inderlig fortrolig . Han ved noie , hvad de hungrige Sjcele havde Behov for , og han forstaar at tale derom i  Kanaans Tungemaal . " Han er ikke tor og langtrukken , men livlig og anskuelig . Talrige ' Skarer flokkedes om ham . Han praMkede ikke blot Ssn- og Festdagene , men var ogsaa forpligtet til at prcedike Mandag og Fredag , og det er et godt Bevis paa den Huuger ,

<h4>1229</h4>Breve at besvare , og naar dette var gjort , forfattede eller oversatte han en eller anden opbyggeltg Bog , saa han altsaa ofrede alle sine Legems- og Sjelskrefter i Guds og sine Medmenneskers Tjeneste . Foruden hans allerede nevnte Skrifter maa anfsres fslgende :  Udvalgte Levnetsbeskrivelser af hellige Sjcele " i 3 Bind ;  Zions Bsrns Harpespil ; "  Guds hellige Kjerliahed og den vanhellige naturlige <b>Kjcerlighed</b> ; "  Aandelige Traktater , " den faakaldte lille Perlesnor . Tersteegen omgikkes ikke med Verdensmennester uden nsdtvungen ; men da vidste han ogsaa at stikke sig saaledes , at de havde den stsrste Agtelse for ham og fik mangen alvorlig Advarsel af hans Omgang . En ihansNerhed boende Vertshusholder sagde engang om Tersteegen :  Naar jeg gaar forbi denne Mands Hus , opstaar der altid en hellig Agtelse hos mig , og Tanken paa ham vcekker ofte hos mig et dybere Indtryk end mangen Prediken . " Forsvrigt holdt han sig under Samver med Verdens Born stille ; men fandt han Anledning til at tale et alvorligt Ord , saa lod han det ikke « benyttet . Herpaa kan Fslgende tjene som Eksempel : Da han engang var paa en Reise til Holland , traf han paa et Sted flere fornemme Herrer samlede ; han satte sig stille ned og lukkede Vinene , som om han sov . Efterat Herrerne fsrst havde talt om allehaande Ting , begyndte de at spille Kort ; da aabnede Tersteegen sine <b>Oine</b> og sagde , at ogsaa han havde smukke Kort i sin Lomme . Da man nu bad ham om at tåge dem frem , tog han op sit nye Testamente . Ved Synet af denne Bog ytrede Herrerne, at denne Bog hsrte til Daarernes Verden ; hertil svarede Legpredikanten, at de vistnok <b>selv</b> vare fulde af Daarskab , gjentog for dem deres tomme , forfengelige Samtale og ssgte med deres egne Ord at overtyde dem om , hvor taabelig de håndlede , idet de med saa unyttige Ting havde forfpildt den kostbare Naadetid . osv. Nogle gave ham sit Bifald , og de Andre bleve idetmindste hmdrede fra sit Forehavende . Altid , naar det gjaldt at forfegte <b>Sandhedens</b> Sag , optraadte han meget bestemt og med den stsrste Frimodighed . Et godt Bevis paa hans Frimodighed have vi i hans  Tanker over den sanssouciske Filosofs Verker . " Denne Filosof var , som bekjendt , Fredrik den Store af Preussen . Hans filosofiske Skrifter , hvori han behandlede Kristendommen mere end übillig , vare blevne tilstillede Tersteegen fra en Ven med Anmodning om , at han vilde udtale sig om deres betenkelige Indhold . Han gjorde dette i ovennevnte lille Skrift . Vi kunne desverre for Pladsens Skyld kun anfsre den hsist interessante Bogs Slutningsord :  O , I Herrer Filosofer af Sanssouci * ) , bliver dog fsrst Filofofer af Grandsouci * * ) — eller det vil gaa eder hsist vedrsvelig . ' Den storhjertede Forfatter er en Fiende af Smigren , en Inder af Oprigtigheo . Han tenker frit og tillader ogsaa Andre en lignende Frihed . Ifald han derfor ogsaa maatte hsre , hvad jeg siger en Ven i Yret , saa er jeg dog rolig for , at han ikke vil optage mine Tanker paa en unaadig Maade . Min sande Hsiagtelse for hans erveroige Person og hans uforlignelige Egenskaber har bragt mig til under Lesningen hundrede Gange at sukke :  , , Hvilket Redskab kunde ikke denne store Mand vere bleven i den store Guds Haand , naar hans udmerkede Forstand , bestraalet af et hsiere Lys , vilde aflegge de hsist skadelige Fordomme mod Religionen , og hans edle <b>Hjerte</b> vilde give alle Kongers Konge og Herrers Herre den Wre , som ham tilkommer . " " Den

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015030308004'target=_>Evers, Ernst, 1879, Rogate</a><h4>508</h4>Midler , var det ikke i <b>Sandhed</b> nsdvendigt at se sig om efter et Stottepnnkt i deres Forladthed ? Men hvilken Virksomhed sknlde hnn vcelge , nåar hun afviste Moderens Forslag ? Naar hnn afviste det , hvorledes vilde Moderen optage det ? Tet bliver hende klart , at den Tid snart vil komme , da hun aabenbart man troede frem i Lyset med sit <b>Hjertes</b> Helligdom , da hendes <b>Kjoerlighed</b> maatte gaa over fra at lide til at kjoempe . Men vil hun blive stcerk nok til at kunne fore en Kamp mod Moderens hele Liv og Stroeben ? Og desnden , nåar en saadan Fremtråden nu bliver uodvendig , burde hun ilke for lcenge fiden have optraadt aabent mod sin <b>Moder</b> ? Maa hun ikke anklage sig <b>selv</b> for Svaghed og fyndig Eftergivenhed ? Ak , min Gud , min Gud , klager hendes Sjel , overgaar det ikke en stakkels fvag Piges Kraft , at ville rao.de og lede en Moders <b>Hjerte</b> ? Er det ikle netop hendes Pligt at folge Moderens ledende Haand ? Et har Elfrieda Treufels allerede loert i Hovedstaden , at beregne Mennestekraft og MennestehjcelP . Ikte saa , at hun ganske lod sin Herre og Frelser nde af Beregningen , men Moderkærligheden , fom har omvcevet hende med et tcet Net og omhyllet hendes <b>Oie</b> med Tange , har bragt hende til at vakle og tage feil af Veien . Barnepligterne ere for nye , for kraftige til ikke at feire , nåar de komme i Strid med andre helligere Pligter . Ikke at hun havde glemt Bsnnen og Pligterne mod sin himmelske Fader , men for Mennesket er Menneskehjertet saa ncrr , og hvor en Moderhaand trcekker kraftigt ned-

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009071303012'target=_>, 1863, Bibelen, eller Den hellige Skrift</a><h4>847</h4>Lidelser erkjender og foler han en Straf af den vrede Gud ; han beder ham , at han ikke vil fortsette denne Straf , ikke . . friste ham over hans Formue " , og netop derfor midt under Lidelserne sijenke ham Visheden om sin vedvarende Naade og deraf folgende snare Forlosning af Trengslerne . — Min Sjlrl er snare forfcerdet ; og du , Herre , hvor llrngc ? Som Grnnd for sin Bon fremstiller han for Herren de forferdelige Virkninger , som Lidelserne allerede have udovet paa hans hele Tilstand ; hans Been ere forferdede ( V . 3 ) , han er ikke blot legemlig i en Tilstand af Zitren og Beven , men ogsaa hans Sjel er aldeles overveldet af denne Folelse . De sidste Ord er en vemodsfull ) Klage , der nesten klinger som en Bebreidelse , saadanne forn oftere i G . T . fremstige af det engstede <b>Hjerte</b> . — Vend om Herre , udfri min Sjlrl ........ thi i Dodcn er der ingen Erindring om dig ; hvo vil prise dig i de Dodcs Nige ? Vi finde her den morke triste Forestilling om de Hedengangnes Rige , som er den fremherskende i G . T . ( Smlgn . herom Job . 3 , 19 Anm . ) . Omendstjont mange Troende i Anelsens Vieblikke hevede sig over den i Visheden om Samfundet med Gud , ligesom Gangen i Jobs Erfaringer viser : saa lunde dog denne Vished , for Forlesningen var fuldbragt , ikke vere nogen stedsevarende , der altid blev sig <b>selv</b> lig , hvorfor heller ikke Udsigten til det hinsidige Liv kunde vere nogen klar og lys . — Hvad der imidlertid paa Troslivets lavere Trin har sin <b>Sandhed</b> henviser til lignende Tilstande , der endnu forekomme ogsaa paa de holere . Naar Herren forlod os i Forfolgelsen , nåar han gav Fristelsen Magt til at ovcrvimde os til Fald : hvo vilde vel da endnu prise ham under Fordommelse ns Tilstand , nåar Sjelen er gaaet tabll ? — Men dette Sporgsmaal er et meget omt , inderligt og barnligt . Det forudsetter , at Gud efter sin store <b>Kjerlighed</b> har en Trang til at forherliges og prises af sine Skabningr , som han har dannet ester fit Billede til Erkjendelse af og <b>Kjerlighed</b> til sig , at denne Trang . hos ham er saa sterk , at han for dens Skyld vil redde og bevare dem . Dette er en stor , herlig <b>Sandhed</b> , der i Forlosningen ved Kristus har faaet sin fulde Bekreftelse ; men kun den har Ret til at gjore den gjeldende , som i Frygt og Beven for Guds Majestet har erkjendt sin Fordervelse og Elendighed . — Hcntcerct af Kummer er mit <b>Oie</b> ; det er bleset gammelt formedelst alle mine Fiender . Ikke som om hans legemlige Elendighed var udgaaet fra dem , men Udtrykket maa forstaaes i den ovenfor angivne Sammenheng . Helbredelsen stulde ste ved Red ^ ning fra hans Forfolgere . — Vigcr fra mig alle I , som gjore Ondt ; thi Herren har hort min lydelige Graad . En herlig Skildring af den troende Bons Virkning . Under den engstelige Klage , der med Fortrostning vender sig til Guds Naade , og uden at de udvortes Omstendigheder i nogen Henseende forandres , modtager den bedende i sit Indre ved Aandens Vidnesbyrd Bsnhorelsens Indsegl og kan nu trostig mode den Trengsel , der dog snart maa ende . — De stulle Vende tilbagc , de maa ophore med sin Forfolgelse . Den Vonhorelse , om hvilken han har faaet en inderlig Forvisning , har en Virkning som om man tral bort et morkt Forheng , der stjulte Fremtiden;

<h4>2278</h4>Vreden maa her — som det Folgende viser — ikke adstilles fra sin Virkning , Mennestets Straf i dets Dodelighed . Her siges ikke : hvo erkjender overhovedet din Vrede , men : hvo giver sig den Moie at 12 erkjende den under Livets Tomhet ) ? — At tcellc vore Tage det lcerc du os , at vi maa bekomme et viist <b>Hjerte</b> . Hver den , som betenker den store Tomhet » i alle jordiske Ting , ja i sil Livs Stolthed , han frygter Gud og styr det Onde , hvilket er Vegyndelsen til al Viisdom . Job 28 , 28. Luther oversetter efter Meningen :  ler os at betente , at 13 vi maa do , paa det at vi maa blive forstandige . " — Vend om , Herre ! hvorlcenge ? — og lad det augre dig over dine Tjenere . Det er et i Skriften hyppig forekommende , Udtryk , at Gud angrer den Straf , han har tiltenkt eller forkynd , et Menneske , nåar nemlig dette af Hjertet omvender sig . 2 Mos . 32 , 12. 14. Ps . 106 , 45. lerem . 18 , 8. 10. 26 , 3. 19. loel 2 , 13. lon . 4 , 2. Am . 7 , 3. 6. Dette maa ilke ansees for et blot mennesteligt Billede , der hos Gud ingen Virkelighet ) har . Naar hans Vrede som hans <b>Kjerlighed</b> er en <b>sand</b> Rorelse hos barn , saa gjor det ham ogsaa for Menneskenes Skyld ondt , at en dem forkynd . Straf stal fuldbyrdes , ifolge den inderlige <b>Kjerlighed</b> , hvormed han <b>moder</b> den Bodferdige . Heraf Udtrykket : 14 „ det stal angre ham over hans Tjenere . 5 Mos . 32 , 36. — Moet os aarlc med din Mistundhcd o . s . v. — „ aarle " ( ordlydende :  om Morgenen " ) opgaar Frelsen ester Natten ; dette skjonne Billede forekommer hyppig i Psalmerne ( Ps . 5 , 4. 59 , 17. 143 , 8 ) . . . Lat » os om Morgenen efter Sorgens lange Nat atter mettes med din Naade . " Det Hebr . Ord betyder ikke blot . . Morgen " men ogsaa . . imorgen . " — 15 Glced os alt efter dc Dage , du plagede os , efter de Aar , vi saa Ulykke . Gled os „ alt efter " kan betyde „ ligesaa meget " og . . ligesaa 16 lenge . " Skjenk os , nu en Erstatning for den lange Tid . — Lad dine Gjerninger aabenbarcs for dine Tjenere . Ved Guds „ Gjer- ' ninger " maa her forstaaes hans Naadcsgjern.nger og Forelser , hans Frelscsgjcrninger . Udtrykket kan aldrig betegne Straffe , da herved steer en Tilintetgjorelse , men aldrig cn Skabelse . „ Guds Gjerning er idel Velsignelse , hans Gang er idel Lys . " Under Straflidelsens Tider er denne Gjerning skjult for Menneskene ; . stjont Guds Forjettelser aldrig kuune slaa feil , legge ' de dog et Dekke for Mennestets Ansigt , da han aldrig seer dem bryde frem i Gjerning . Maaste maa her nermest tenkes paa Indtagelscn af Kanaan , da nu de Strassedomme , hvormed Gud havde truet , vare tilende . Hertil synes Udtrykket „ din Herlighed over deres Born " godt at svare , saasom af de Gamle endnn mange , hvoriblandt Mose <b>selv</b> maatte do , hvilke vel kunde se . . Guds Gjerninger " i hans Ledelse af Folket indtil den Tid men ikke hans 17 Herlighed ved Erobringen af Landel : — Herrcus Vor Guds Liflighcd er al den Salighet » , So , dhed og Velgjorenled for de syndige Mennesker , der ligger i Guds Egenskaber , alle Krefter og Virkninger af , alle Panter paa hans Naade . — Verre over os d . e . fra oven ; han holde sin Haand over os og hvad vi i stor Skrobelighed herueden udrette i hans Tjeneste det befeste og grnnde hans almeglige Haand / saa der kommer noget Bliveude ud deraf . Udtrykket „ vore Henders

<h4>3242</h4>Son bereder sin Fader stedse tilbagevendende Ulykke ; en saadan Ulykke , der drager sig gjennem det hele Liv , er ogsaa en trettetjer <b>Kvinde</b> . Hoist treffende og anskuelig sammenlignes hun med det uophorlige Tagdryp . — Huus og Gods n Arv fra Foreldre ; men fra Her- 14 ren lommer en forstandig <b>Kvinde</b> . Formue kan vel ogsaa falde i et uverdigt Menneskes Lod , men en forstandig <b>Kvinde</b> kan han ikke saa let faa , thi hun er en Gave fra Herren . Dovenstnb ncdseuker i dyb Sovn , bringer Mennestet til at stivne , gjor ham foleslos . Ten , som forbarmer sig over den Arme , laaner Herren . Han 17 glver Me sine Ting bort , thi de Fattige anseer Herren fornemmelig for sine , og han berigtiger den Gjeld , som de ikke . kunne betale . Tugt din Son ..... . men til at drebe ham maa du ei 18 lade dlg henrive , nemlig til ved Heftighet ) at knekke hans Mod . Mange Planer ere der i en Mauds <b>Hjerte</b> ; men Herrens 21 Raad , det stal beståa . Utallige Anslag og Forsetter gaa tilgrunde , medens Guds Naad alene bliver staaende . C . 16 , 1. Menneskets Mistundhcd er hans Glede , og lykkeligere er den 22 Arme end Logncren . Den sande <b>Kjerlighed</b> , der aabenbarer sig i Velgjerninger bereder de Fromme den storste Lyst ; er nu ogsaa en Mand fattig , men har dette Sindelag , saa er han lykkeligere end Logneren , som ved Falskhet » - negler den Fattige sit Gods . Herrens Frygt er til Liv , og met gaar han til Hvile 0. s . v. 23 I Gudsfrygt ligger en rig Nering for det indre Menneske , den er et Livsens Tre ( C . 3 , 18. ) der holder ham oppe . Den Lade stikker sin Haand i Fadet 0. s . v. Hau styr den 24 ringeste Umage , for at erhverve sig Brodet . E . 26 , 15. Slaa Spotteren , saa stal den Uvittige vorde klog ; og viis 25 den Forstandige tilrette , saa vil han komme til Indsigt og Knndstab. Meningen er ikke den , at Spotteren ved Slag stal blive viis . Tverlimod er derom denne , som <b>Sandhedens</b> bitreste Fiende intet Haab ; men andre blive derimot ) kloge , nåar de merke det Alvor , hvormed man optreder mod saadanne . Hvad der nu i dette Fald kommer andre tilgode , nyder den Forstandige ogsaa ved en ringere Tilrettevisning for sig <b>selv</b> . Lad af min Son ! at hore paa Formaning for at forvilde dig 27 bort fra Kundskabs Ord . Verset lerer paa ironisk Maade : hold kun forst op , saa vil du snart se , hvad Folgen bliver . Et nederdregtigt Vidne spotter Retten , og Ugudeliges Mund 28 fluger Uret . Saa meget det staar til ham , fornegte . han Netferdigheden, forstyrrer Retten paa Jorden , netop ligesom et ryggeslost Menneske drager Ulykken til sig , som kunde han aldrig faa nok .

<h4>3531</h4>end for , — eller , at Verset maa noie forbindes med det Foregaaende , der uden dette ikke vilde have nogen Slutning . Denne sidste Opfatning er maaste den rigtigste . Hyrdepigens Klager , over at bun , endnu ikke har faaet Deel i det belovede Samfund med ham , da hun ikke veed hvor han , Kongen , nu som Hyrde har opslaaet sit Paulun . De osterlandste Konger have oftere robet en Tilboielighed til at nedstige fra sin kolde og ensomme Hoihed for at blande sig mellem de andre Menncster og tåge Deel i deres Gjerning . Bruden narer Bekymring for sit Rygte , nåar hnn stulde gaa og soge efter ham . De berygtede <b>Kvinder</b> pleiede at gaa med Slor ( se 1 Mos . 38 , 14. 15. ) . — I sin hele Aabenbarelse omgaaes Herren med Menneskene som deres Lige ( smlgn . 1 Mos . 18 , 17. 18. 4 Mos . 12 , 7. ) . Io mere personlig og menneskelig , — desto guddommeligere er hans Omgjangelse med dem . Hver den , som har smagt ' hans <b>Kjarlighed</b> , langes efter en saadan . Intet er imidlertid mere almindelig . , end at  Medbrodrene " miskjende en saadan Kjcrrligheds Minger . Ved „Medbrodrene forstaaes her de , som i det ndvortes Vasen have en vis Lighed med den uforlignelige Brudgom . Veed du det ikke , du stjonncstc blandt <b>Kvindcrnc</b> ! saa gak du 8 ud i Faarcncs Fodspor og vogt dine Kid ved Hyrderncs Boliger . . — Den Elskende viser hende hvad bun stal gjore . Hun stal knn <b>selv</b> som Hyrdinde gaa ud med sin lille Hjord og opholde sig ved Hyrdeboligerne, hvor ogsaa han vilde vare . Menigheden sinder sitt Herre og Hyrde sikrest og uden Fare , nåar den soger ham i de Gjerninger , hvori han lever paa Jorden , — nåar den vogter sine Faar paa de sadvanlige Stier . Med mine Heste for Faraos Bogne ligner jeg dig , min Ven- 9 inde . Det af den Elskede udtalte Duste har bragt hende Kongens <b>Hjerte</b> saare nar ; — han forherliger hende ved en Lovtale . Salomo havde ladet fore Heste i stort Antal fra det paa disse Dyr saa rige Wgypten . Maaste havde batt modtaget dem som Gave af Farao , hvis Datter var hans Dronning ( 1 Kong . 3 , 1. ) . Han har maaste havt enkelte sjelden smukke Dyr , der her kunde tjene ham til Sammenligning, men man behover ikke at antage dette , da Heste endnu vare Sjeldenheder i Jerusalem . — GuldkjaHer ville vi gjore dig il mcd Solvprikkcr i . Det Smykke , som Hyrdepigen stal faa , er herligt og i <b>Sandhed</b> kongelig . . Naar hun forst horer ham til , smykker Herren hende som det anstaar hans Brud . Saalcengc Kongen var i fin Krcds , gav min Nardus stn Lugt . 12 De have indtaget et Maaltid ved hinandens Side . Med Forbauselse udtaler hun nu , hvorledes hendes kostbare Salve ei . bleven ligesom belivet ved Kongens Narhed , hvilken fljon Duft den har sendt ham imode . Det er en Virkning af Optagelsen i det fuldkomne Samfund med Gud , at alle ncttnrlige Gaver , der allerede findes i Menigheden , faar en ny Rod og Kraft . — Min Elskede er mig en Myrrha- 13 tugle , som 0. s . v. Min Duft holder hende bestandig fast i Erindringen om hvor og hvad hun er . — Min Elskede er mig en 14 Cypcrdrue i En-Gedis Viingaarde . „ Eyperdruen , " af Araberne

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111103032'target=_>Krummacher, F.W.., 1856, Thisbiteren Elias</a><h4>2153</h4>hvilke Ord ! Hvorfra komme de ? Hvo udtaler dem ? Nu , en Begynder i Kristendommen , skulde man mene ; et Menneske , som endnu ikke er vis paa sin Naadestand , eller endog En , som endnu ikke er troengt igjennem den sncevre Port . Men ikke saa , mine Kjcere . Disse Ord udtaler en Mand , der staar paa en HKdc af Trocsliv , som saudsynligvis ingen iblandt os . Dem udtaler den samme Apostel , som turde rose flg :  3 eg lever , dog nu ikke jeg mere , men Kristus i mig . " Dem udtaler den Helt , som vidste , paa hvem han troede , og kunde juble :  Hvo vil stille mig fra Guds <b>Kjcerlighed</b> ? " Den Hellige , i hvem Kristus hnvde vundet en Skikkelse , som hos ingen Anden ester ham , og som hverken saarede BcstVdenheden eller <b>Sandheden</b> , naar han udraabte :  FFlger mig efter , og ser paa dem , som vandre saaledes , ligesom I have os til Forvillede." Den Mand , der straalede som den mest glimrende Stjerne paa den unge Kristenkirkes Himmel , han bekender her frit og aabent , at heller ikke han endnu har grebet det .  Nei " , siger han ,  jeg er endnu ikke fnldkommm " , og gjentager det senere med sorstcerket Eftertryk :  leg agtcr ikke mig <b>selv</b> at have grebet det . " I studse mine Elskede , I ere forundrede . la , vcerer det kun , dog vcerer det uden Misforstand . Paulus taler her ikke om det , som han er i Kristo for Gud . Naar der handles derom , saa ftrer han vistnok et andet Svrog . Da hcdder det :  Hvo vil anklage ? Hvo vil fordMme ? Viere blevne retfcerdige ved Troen ; med et Offer har han for stedse fuldkommet os " . Apostelen taler om sit <b>Hjertes</b> Tilstand . Han taler om sin indre Stilling til Herren , om sit Troesliv , og kun med <b>Blikket</b> derpå a tilstaar han uforbeholdent , at han endnu ikke har grebet det , at han endnu ikke er fuldkommen , eller , forn Ordene bogstavelig lyde , at han endnu ikke ser sig ved Maalet .

<h4>2446</h4>engang hcendes disse Folk , at de en Tidlang maa ombytte deres indre Glceders fedeste Grcrsgange med den aandelige Armods Fde vinterlige Drken , saa beklage vi dem dog af den Grund ikke . Det er dem gavnligt . Noget Nndvcerligt bliver uuddraget dem ; maatte de istedet derfor finde og lcrre at holde fastere ved det Uundbcerlige og Vcesentlige . Vistnok ligger nu for en Tid deres Fredshus i Ruiner ; men hvad betyder det , naar det igjen opbygges paa mere urokkelige Grundvolde ? For et Meblik synker og falder det sammen ; men er det Skade , naar en raadden Krykke bliver slaact bort fra dem , for at de nu skulle stMc sig til en DiamantsMe ? — Dog en GloedesfMse gives der i Herrens Samfund ; den , forn faar famme at nyde , han spge at beholde , hvad han har ; det er en Velsignelse . Denne Glcrde har endnu ikke bragt Nogen Fare . Den er meget mere den Kristnes Helligelfe og hans Styrke . Den er <b>Moder</b> til det Skjønneste , som nogcnsindc blomstrede i Mennefkehedens Have . Den felvforncegtende Kjcerlighcds mest glimrende Handlingrr flyde af denne hellige Kilde . Denne Glcedesblomst bliver ikke plukket i Selvbcskuelsen . I Betragtningen af Kristus og af det , som han er blevm os , aabner den os sin Kalk , og dens Duft er et himmelsk Livs Krceftcr . Ham , Livets Fyrste , ser man i Aanden . Som et aabent Skatkammer staar hau for os . Man er henreven af hans Lidclscrs Herlighed , af hans SkjMhed forn Forsoner og Midler . Man ser da ikke paa sig <b>selv</b> , man forglemmer derover Alt . Hans Forbarmelscs Afgrund aabner sig for os . Man h ^ rer <b>Kjcerlighedens</b> Ocean bruse i hans Varm . ForlMningsplanens Dybheder aabncs , Sløret vigcr fra det Hemmelighedsfuldc ved hans Fyldestgj ^ relfe for os . For vort TroesM udfolder sig glimrende den Hcrlighed , som han erhvcrvede os . Man ser sig i hans Blod renset fra Synd , i hans fuldkonme Betaling befrict fra al Skyld , i hans Lydighed hellig og rctfcerdig ; for stedse fuldendt i hans ForbM; — indfat ved ham i den Barnerettighet » , som Serafen ved hans Throne kunde misunde os , og optaget i en Gudskjcerligheds Skjß , forn ikke i Noget er forskjellig fra den , hvormed den evige Faders <b>Hjerte</b> belger sin Enbaarne felv imFde . Hvo skildrer nu den inderlige Henrykkelse , som overvelder os ved dette <b>Blik</b> . Odu falige Liv i en faadan Beskuelse af den store Ippersteftrcest ! Jordens StM er rystet af Fodcn . Sjelen straaler i herligt Festdagssmykke . I denne Stad er Troens

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011051004069'target=_>Ewald, H.F., 1875, skotske Kvinde paa Tjele</a><h4>310</h4>~ Margaret ! leg talte nu saa lcenge med en Andens RO og dog ud af mit eget <b>Hjerte</b> ! I tillade mig nu at sige Eder , hvad der har sig der og groet en langsam og sikker Vcrxt i tyende samfulde Aar . la , I var ncesten et Barn , da jeg saae Eder ftrste Gang ; vi legede sammen , og jeg blev Barn igjen ; med Uskylds Leg kom den <b>Kjcrrlighed</b> i mit <b>Hjerte</b> , saa I maa vide , at den er reen og tro , som Guld . I sige mig da nu — under I mig saa vel , som Blanseflor undte Flores ? " Margaret blev som et dryvpende Blod , og saa , blev hun bleg som enLillie . Hvad hun ofte havde forestillet sig i sine fantastiske , vaagne DrMme , det var nu virkelig steet . Hendes dybe <b>Kjcrrlighed</b> gjennembcevede hende , og hun formaaede i det fyrste Aieolik ikke at svare . ~ Margaret , " sagde Folmer , idet han greb hendes Haand , ~ har jeg feilet i at tcenke , at Eders Hv staaer til mig ? Er jeg den Mand , I vilde scette Tro og Love til ? " ~ I er saa ! " svarede hun , medens hendes taareblcendede <b>Blik</b> og Mmc Smiil tilfulde bekrceftede hendes Ords Oprigtighed . ~ Gud have Lov ! " Folmer og vude drage hende til sig , ~ da er I nu min Fcestemjl ! " Dog , hun stMe ham fra sig og blev saare bleg : thi hans Heftighed bragte hende pludselig til Besindelse . Netop som hendes <b>Kjcerlighed</b> udfoldede sig l : g en deikg Blomst med berusende Duft , ståk Slangen sit Hoved frem og hun gyste . Hendes Fasters forblommede , giftige Ord randt hende i Hv , og alle de bittre Tanter , de havde avlet , og som en Stund vare forglemte , mylrede frem kg en Snogeyngel . ~ Eders sagde hun med usikker RO , ~ er jeg , det I fuldt vel maa vide , paa ingen Viis , M Fcestensringen er sat paa Eders og min Finger for Guds hellige Alter . " , ~ <b>Sandt</b> nok det , " svarede Folmer og holdt hendes Haand fast : ~ og dog har Gud Herren <b>selv</b> feet og fornummet, hvad her mellem os er foregaaet . Er dog Me

<h4>685</h4>~ havde ikke Uret : Leiligheden var übekvem : dog , mit <b>Hjerte</b> brcrndte i mit Bryst . " ~ Den Ild vil vel slukkes , " sagde Margaret med et Smiil og taarefyldte ~ og en anden tcendes : mv drager I toenkeligt snart i Leding og fcerdes langt fra mig . " ~ I sige ikte Sligt ! " sagde Folmer harmfuldt . ~ Det er , som yadede I Eder <b>selv</b> . " ~ Og om saa var ? Et UlykkensVarn er jeg , Folmer ! Den stal have den kranke Skjebne , der ei den gode kan faae . " Folmer saae paa hende og fornam , hvor fast hans Hierte var oundet til hende : ingensinde , der hcende hvad der vilde , kunde det elstede Aasyn , som nu var ham saa ncrr , vorde glemt af ham , drog han end nok saa langt bort ; ingensinde vilde Gjenlyden af denne Stemme , hvis Tone , trods Ordenes haalMe Bitterhed , i sin Skjelven forraadte den dybeste <b>Kjcerlighed</b> . uddpe i hans <b>Hjerte</b> . Han ftlte , at hun var hans og ingensinde vilde formaae at give sit <b>Hjerte</b> til nogen anden saa stor Trofasthed og HMndethed lyste ud af hendes deilige Aasyn : men dog indgpd den mMc Ild , der nu funklede i hendes <b>Blik</b> , og som svarede kun altfor godt til Ordenes HaablFshed, ham stor Bekymring : thi den erindrede ham om hendes aandelige Vildfarelse og om den skjebnesvangre Omstcendighed, at han efterlod hende i en rcenkefuld Munks aandelige Varetcegt . Vel dog for Folmer , at han ikkun tcenkte vaa den aandelige Skilsmisse , Margarets svcrrmeriste , vavistiske Overtro kunde bevirke , og at han endnu ikke formaaede at lcese med Klarhed i det Helvedes Dyb , Broder Vigand kaldte sit <b>Hjerte</b> . Han blev vakt af sin Dvale ved Knuds Tilraab , at nu gik Dandsen til Brudekammeret . Folmer saae , at det forholdt sig rigtigt , og at Rigens Hofmester end ikke manglede i den store Skare . ~ Nu gaae vi med i Rad , " sagde han til Margaret , ~ og saa dandse vi derefter til den lyse Morgen . " „ Gjerne det ! " udbrß Margaret og reiste sig brat , ~ o , kunde I , saa <b>sandt</b> , dandse mig til Di ^ de ! "

<h4>1303</h4>Medens Knud fra det stille Kloster drog ud i Verden , fuld af HM Mod og fremdeles rede til at lße Panden mod enhver Muur , som hans ulykkelige Skjebne vilde reise paa hans Vei , blev hans <b>Moder</b> og SOer tilbage , glcedende sig ved Klosterfreden . Med ufkrMtet Andagt bivaanede de Paaskefestens HMd , hvis Helllgholdelse paa papistist Viis afgav et vexlende Skuespll , rigt paa Sandserne maatte bedaares og al Eftertanke forjages under denne indviklede Afgudsdyrkelse , som i Aarhundreders Lpb var bleven sat i System , paa een Gang en Frugt af hierarkisk Snildhed og en barnagtig Fantasi , ret stikket til at blcende vankundige Sjcele . Og hvad var den svagsindede , af en haard Skjebne kvede skotske <b>Kvinde</b> og <b>selv</b> hendes HMndede Datter nu Andet , end vankundige Sjcele , ssgende efter <b>Sandhedens</b> Lys der , hvor Menneskenes taabelige Paafund ncesten ganske det ? Med blid Hengivelse til det Mystiske og med den Umyndiges Fred i Hjertet bivaanede Genete Kirkeceremonierne , medens Margaret med mld Lyft fordybede sig i Andagt og spgte Forglemmelse : et fanatisk Svcermeries Afgrund . Fordybet : Helgentilbedelse saae hun ikke tilbage , i det forfoengelige Haab , et denne Syndflod af BMner , Sukke og Kys paa Tr « og Steen vilde formaae at udslette Fortidens Erindringer og hendes <b>Hjerte</b> for jordisk <b>Kjcerlighed</b> og jordist Lykke . Det var nu saare langt fra , at Anfegtelsernes T : d skulde vcere omme for Margaret , tvertimod havde Knuds og Hans Thordsens Beftg nys avlet urolige Tanter i hendes Sjcel , men saa lcenge HMden varede , lykkedes det hende at holde dem nede . Gudsdyrkelse i den Stiil , som hun nu blev Vidne til her i Klostret , var

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100803022'target=_>Kennedy, Grace, 1850, Miss Grace Kennedy's samtlige Værker</a><h4>1329</h4>gode Vrempel for <b>Die</b> , som er den fornemste Fordeel af den huslige Opdragelse , saa gav min <b>Moder</b> endelig efter , efterat min Fader havde lovet at opfylde heudes Vnste om at jeg ikke skulde sendes til en Skole , hvor der var et altfor stort Antal Drenge ; dog forbeholdt han sig Valget . Forstanderen for det Institut , som han valgte , besad alle de Egenstaver , der vare nødvendige , med Undtagelse af en eneste , som min <b>Moder</b> satte hsiest . Han var en Mand af store Talenter , grundig Lcrrdom , og en fim og dannet Smag ; han havde ogsaa vcrret meget iblandt Menneskene , og havde derfor en Werdensmands elegante Manérer . Ha » modtog blot et bestemt Antal af Pensioncrrer , og Nogle af mine Kammerater vare af Landets forste Familier . Under en saadan Larer , der ganske offrede sig for sine Elevers Uddannelse , og forstod at vcrkke min Interesse for de Studier , som han <b>selv</b> satte meest Priis paa , gjorde jeg saa hurtige Fremstridt , at jeg snart erhvervede mig hans Gunst i en hoi Grad . Det laae ogsaa i min Charakter at strcrbe efter at blive elsket af dem , som omgave mig ; derfor erhvervede jeg mig snart ogsaa mine Kammeraters <b>Kjcrrlighed</b> . Kom jeg hjem , saa blev jeg smigret fra alle Kanter ; jeg opdagede , at jeg var min Faders Stolthed og Haab . Min Moders Kjcrrlighcd til mig syntes at have voret under min Fravcrrelse . Hun ssgte dog paa det Omhyggeligste og Vmmeste at overbevise mig om , at ingen Kundskaber , ingen Dannelse , ingen Agtelse , <b>Kjcrrlighed</b> og Beundring , som Andre kunde vise mig , havde nogensomhelst Vcrrd uden Religion . Hun sagde , at Alt dette var kun Ting , hvilke Djcrvelen brugte som Midler til at ncere Stoltheden og Hoffcrrdigheden i det menneskelige <b>Hjerte</b> , og lokke Sjcelen i den evige Fordervelse . Jeg hsrte paa hende , fordi jeg elssede hende , og ikke vilde foraarsage hende nogen Kummer . Min Lcrrer var alligevel en Deist , og jeg havde lcrrt af ham at bctragte alle Neligionssystemer , enhver Religion , undtagen den naturlige , som Prccsteopfindelse og Bedrag, hvoraf kun svage og « vidende Folk lode sig bedaare . Jeg blev i denne Pensionsanstalt indtil jeg drog til Universitetet . Du kan neppe tcrnke dig et Menneske , kjerre Katharine , som kan vcere mere übekjendt med Werden og den mennesteligc Natur ,

<h4>2996</h4>Tjenere ? Hvorfor erklcere vi , at vi troe paa hans Viisdom , <b>Sandhed</b> , Omhu og <b>Kjcrrlighed</b> , men frygte alligcvcl for at modtage Beviserne derpaa ? Hvis vi virkelig troode , saa vilde vi falde ydmyge for haus Fodder , og sige : Herre , jeg vecd , at jeg ikke kan lede mig <b>selv</b> ; du alene formaaer at rense mit <b>Hjerte</b> og forberede mig for din Himmel ; o , saa tag da mit <b>Hjerte</b> , og omform det i dit Billede efter din Biisdom ; du kjender alle mine Fristelser og Svagheder ; o , saa styr da mit Liv saaledes at jeg kan blive vcrrdig til den evige Salighed i dit Samfund ! Hor ikke mine Bonner , hvis jeg beder om noget Andet , end hvad der tjener dertil . Havde vi en saadan Tro , da vilde Alt , hvad der <b>moder</b> os , synes os sendt af den Wiisdom , der aldrig kan feile , af den <b>Kjcrrlighed</b> , der er dybere og uudforskeligere , end vi kuune fatte det , ommere , end vi kunne fole det , da vilde vi modtage Alt , hvad han sender os , som det Bedste og Gavnligste , om det end maatte synes os nok saa uheldsvangert ; havde vi denne Tro , da vilde vi erfare , hvor kraftigt hau formaaer at troste os <b>selv</b> i den bittreste og haardeste Trcrngsel . "

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016101748015'target=_>Calman, A.L., 1876, John Ashworths Liv og Virksomhed</a><h4>601</h4>med det Gode se til at faa ham til at gaa did , og hvis man blot tager Sagen fat paa den rette Maade , vil man nok se , at man faar sin Hensigt opnaaet . Saa raader jeg ogsaa indstændigen til , at man begynder at lcese den hellige Skrift hjemme i Husene . Dette bringer mig til den sidste Del af det , som jeg onster i nærværende Foredrag at komme til , nemlig Moore . Det er den allervigtigste Del , og jeg maa tilstaa , at jeg aldrig kommer det noer , uden med Mrefrygt og Mistillid til mig <b>selv</b> . Thi det er ingenlunde en let Sag at raade med Hensyn til Opdragelsen af Born , og i mange Familier finder man Karaktererne saa forskjellige . Det , som vilde virke fuldstændig knusende for et Barn , vilde neppe gjore det mindste Indtryk paa et andet . Hvilken Visdom , hvilken Dommekraft , hvilken " Fasthed fordres derfor ikke ? Hvis der er nogen paa Jordens Overflade , som treenger Herrens scerlige Bistand, saa er det Moderen til en Borneskare . Og hvilken deilig Lyd ligger der ikke i Modersnavnet ? Der er tre Ord : <b>Moder</b> , Hjem og Himmel — hvilken himmelsk , hvilken velsignet Lyd de give ! Hust paa , at aldrig nogen Nation hæver sig over sine <b>Modre</b> ! Hver Nation bliver nemlig netop til hvad dens <b>Modre</b> gjor den til . Gud har nemlig lagt et Kjærligheds Dyb i Dit <b>Hjerte</b> , Marie , som kun staar tilbage for Hans egen <b>Kjærlighed</b> . Og Han har givet Dig en Magt , som aldrig kan bestrides med Ord . Naar derfor et Barn bliver lagt i Dine Arme , Marie , saa stal der fra Dine Lceber udgaa Ord , der i den lille übodelige Sjæl vil vise sig virksom i god eller ond Retning for Tid og for Evighed . Lamartine har sagt : „ Den Haand . der passer Vuggen, behersker Verden ! " og det er <b>sandt</b> . Voer derfor

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012070608274'target=_>Bernhard, Carl, 1878, Krøniker fra Kong Kristian den andens Tid</a><h4>897</h4>Sommerdag , som den svdlige <b>Kjcerlighed</b> . . . Saa ved jeg ilke , hvad vi stulle forkorte Tiden med , til Torben Oxe kommer. Du bliver rod , Birgitte . Er du itke mere Herre over dit Ansigt ? Og er den flygtige Slotsherre endnu bestandig i dine flygtige Tanker ? Svar mig ikte , det er unodvendigt , dine Kinder have alt svaret . " — „ leg forsikrer Eders Naade ... " — „ Og jeg forsikrer Eder , Jomfru , at jeg ilke tror et Ord af Eders Forsikringer . Fy , Birgitte , har du faa liden Tillid til mig , at du endogsaa vil ncegte , hvad jeg godt kan se . Tro mig , Pige , jeg er for meget <b>Fruentimmer</b> til at lade mig den Snak bilde ind om din Uovervindelighed og Ligegyldighed mod Herrernes Opvartning , som du har faaet hele Verden til at tro , som dens Oontiteor . Ligesom om du ikte var sexlen Aar ! " Birgitte Bryste smilede og sagde : „ Naadigste Dronning , endnu har mit <b>Hjcerte</b> ikke talt til Fordel for nogen af vore unge Herrer . Skulde det ste , da stal I vcere den forste til at erfare det . Men jeg tvivler ncesten om , at det nogensinde vil ste , thi jeg forlanger megen <b>Kjcerlighed</b> for at elste igjen , og <b>sandt</b> at sige forekommer det mig , at vore Herrer maa tun have lidet tilovers til os , eftersom de elste sig <b>selv</b> saa hojt . Den Mand , som jeg stal stjcenke mit <b>Hjcerte</b> , han faar det helt og udelt , og han maa for mig kunne glemme sine Smaatilbojeligheder , og fremfor alt sin Forfcengelighed. Men det kan vist nok ingen af dem , jeg endnu har fet . Min Uovervindelighed , som Eders Naade taler om , har mere sin Grund i Mcendenes Svaghed , end i min egen Styrke .... thi jeg er vel ikke stabt mere uden <b>Hjcerte</b> end andre <b>Kvinder</b> , " tilfsjede hun med et lille Suk , fom hun dog sjebliklig kvalte for at sige med et Smil : „ Tit og mange Gange forekommer det mig dog , som om det ganste mangler mig — og jeg kan ikke sige , at jeg er bedrsvet derover , nåar jeg ser , hvor megen Kval andre have af deres Hjcertesanliggender . " — „ Vogt dig , Birgitte , " sagde Dronningen . „ Du tcemper allerede med dig selo . Io lcengere du kan uoscette denne Kamp , jo farligere bliver dens Udfald . Vogt dig ! "

<h4>1469</h4>Hcenderne i Skjodet , som hun sad i dybe Tanter , og stirrede lige ud . for sig . — „ Hvad tcenker du paa , Dyveke ? " spurgte Kongen pludselig . „ Er det paa Slotsherren ? " tilfsjede han med et skarpt <b>Blik</b> . Uagtet Dyveke var letsindig og manglede Karakterfasthed , havde hun dog saa megen Selvbeherstelse , at hun kunde skjule , hvor meget dette Sporgsmaal overrastede hende ; hun siiftede nceppe Farve , idet hun svarede : „ Naadigste Herre , hvor salder I paa , at jeg nu skulde tcenke paa Torben Or.e ? " Kongen lod sig stusse af denne tilsyneladcnde Rolighed , fom ncesten udtsmte hendes Krcefter . Han vidste ikke , hvor voldsomt hendes <b>Hjcerte</b> bankede i dette Djeblik . — „ Ha ! Dersom Hans Skriver har bedraget mig ... detsial tomme hamart at staa , " sagde han halv sagte , som ved sig <b>selv</b> . — „ Hans Faaborg ? Den forrcederste Skjcelm ? " udbrsd Dyveke , som begjcerlig greb en Anledning til at turde vise den Bevcegelse , hun var i . . . Naadigste Herre , er jeg da sunten saa dybt hos Eder , at en saadan Usling tor bagtale mig for Ever ? Men det er Hcevn , thi ban har vovet at anmode mig om <b>Kjcerlighed</b> , og jeg har afvist ham med Foragt . Og han tor bagtale mig for Eder ... og I troer en saadan nedrig Forrceder ! Alle Helgene ! " ' glennem Kongen ; han fcestede et stift Bltt paa hende , og hun udholdt det , uden at slaa Bjnene ned thi hun talede jo <b>Sandhed</b> , i det Djeblit hun anklagede sm Mnde . Kongens Bjne rullede ustadig , han bed sig Laben til Blods , men tav .

<h4>1689</h4>Kjcerlighed , og Gundel maaste efter at hore denne Tilstaaelse , vilde af dem have undvceiet den store Nydelse , at udtrykke deres Fslelser i det daglige Livs Talemaader og omdanne de almindelige Scetninger til Hjartets glsdende Sprog , forstaaeligt og Yndigt for de indviede , og tomt og indholdslsst for ethvert fremmed Vre . HosKlavs Daa var denne <b>Kjcerlighed</b> ovstaaet pludselig , ncesten ved det fsrste <b>Blik</b> , som det faa ofte sier hos Mcend , der ere begavede med et varmt <b>Hjcerte</b> og en levende Indbildningskraft. Gjennem Djet havde hendes Elflvcerdighed fnndet Vejen til hans <b>Hjcerte</b> , og dette var maaste Me siet , hvis Gundel ikte havde vceret faa smuk , at han var forelstet allerede den fsrste Gang han faa ' hende . Hendes <b>Kjcerlighed</b> derimod var vozet langsomt , nceret af langt aandeligere Fode , fom det i Almindelighed er Tilfceldet med <b>Kvindens</b> <b>Kjcerlighed</b> , der ikte engang i sin Fremspiren er saaledes knyttet til det ydre , som Mandens . Hun er fra den tidligste Barndom vant til at vise sine Tanter og Folelser tilbage i sin egen Barm , de faa en gansie anden Betydning i hendes stille , enssormige Liv , og nåar de omsider henvendes paa den Gjenstand , om hvem de saa ofte og faa ssdt have dromt , da voxer <b>Kjcerligheden</b> langsomt og klarer sig <b>selv</b> til Bevidsthed , idet den atter gjennemgaar de ofte dromte Stadier , Skridt for skridt , indtil den omsider nåar en Styrke og en Renhed , som maaste kun <b>Kvindens</b> <b>Kjcerlighed</b> er i Stand til at opnaa , - og som Mandens <b>Kjcerlighed</b> itte altid er i Stand til at paastjsnne . Dette var dog ingenlunde Tilfceldet med Klavs Daa , thi han baade fattede , paastjonnede og gjengjceldte Gundels <b>Kjcerlighed</b> . Og i deres stille Tanter dromte de sig begge den lytteligste Fremtid , idet alle de udvortes Betingelser for en lykkelig Forbindelse vare ens . I Fru Anne Arvidsdatters aabenbare Godhed for den unge Mand troede de at se en stiltiende Billigelse af den under Modersjet fremspirende og voxende <b>Kjcerlighed</b> , og en Erkjendelse af , at det faderlige Samtykke ikke vilde blive ncegtet . At Fru Annes Lcerdom hverken tillod hende at hore eller se , faldt ingen af dem ind med en Tanke . —

<h4>2103</h4>— „ Saa vidt tan Angst og Skrcek drive disse hovmodige/' sagde hun spottende . „ Dronningens Jomfru taler om at kncele for den hollandske AZblekjcelling ! Men Wvlekjcellingen stal lcere Eder , hvad Hcevn er , nåar man har opsat den i mange Aar . og altid har dojet Uret , " vedblev hun med en Heftighed , som bestandig tiltog . „ Nej , vil I forbede Torben , da maa I gjore det hos andre end hos mig . Thi det siger jeg Eder , at fsr stulde jeg bide min Haand af , for jeg skulde udstrcekke den mindste Finger for at afvende det Svcerd , der hcenger over Torben Oxes Hoved . Og mindst nu , da jeg ved , at han har elstet Eder over Aar og Dag , medens han har drevet Spot og Spe med den , som var Kjod as mit Kjod , og Ben af mine Ven . Han har rammet mig i min eneste , eneste <b>Kjcerlighed</b> , og han har knust mit <b>Hjcerte</b> . . . Jeg ved nu , at jeg kan ramme ham i hans <b>Kjcerlighed</b> og gjore ham Gjengjceld , og jeg stulde ikle gjore det ? Nej , saa <b>sandt</b> jeg er en barnlos <b>Kvinde</b> , ... saa <b>sandt</b> mit eneste Barn blev mishandlet i sin Hjcerterod , ... saa <b>sandt</b> hun blev handlet som en Trcelkvindes Barn , . . saa <b>sandt</b> hun blev skammelig myrdet ... Nu stal Gjengjceldelsen komme ! " — Sigbrit stod som en Besvcergerinde for den tilintetgjorte Birgitte . Med hcevct Arm udtalte hun Fordommelsen over Torben Oze > hendes Ord lode som Dolkestik , — og de gjennemborede Birgitte Brystes <b>Hjcerte</b> . Hun vaklede og var ncer ved at segne omkuld . En ojeblitkelig Medlidenhed med den unge Pige rsrte sig i Sigbrits haarde , af Sorg og Fortvivlelse end mere udtorrede <b>Hjcerte</b> . Hun udstrakte Haanden for at underskotte hende . Men da Birgitte Bryste Me kunde tilbageholde et Udbrud af Forfcerdelse , vendte atter Vreden tilbage i den hcevngjerrige <b>Kvindes</b> Sjcel , og ligesom for at hcerde sig <b>selv</b> mod et lignende Nnfald , sagde hun : „ Saa nng og saa smnk var Dyveke en Time fsr hun dsde , ... og dog drcebte de hende ! — Nej , det stulle I ikte have gjort mig omsonst ! Gaa ! Vi have intet mere at tale sammen ! " Med en bydende Bevcegelse med Haanden viste hun Birgitte Bryste Doren , og vendte sig stolt fra hende .

<h4>2788</h4>Oriis vil lcere , af hvad her er stet , at man itke stal anvende til . det , som mest as alt bsr vcere frivillige . " Gundel betragtede ham <b>kjcerlig</b> . „ Eller ? " sagde hun psrgende , da han tav . - z - „ Eller jeg vil bringe hende til min <b>Moder</b> paa Vurre , " sagde Klavs Daa . „ leg vil sige til hende : Jeg ) ed vel , at der har vceret Kiv mellem to Fcedre ; men Varns Lykte er vel meie vcerd , end en gammel Strid mellem mlige Mcend . Denne Pige har stjcenket mig sit <b>Hjcerte</b> og in Tro , jeg har tåget hende fra hendes Faders Hus og fra hendes Moders Arme . Jeg beder om Plads for hende ) ed Eders Arne , om den Plads , som den vilde ilke ncegter ) en forfulgte , , nåar han kommer til ham i Drkenen , var end hans Dodsfjende , som bad derom . Jeg beder Eder it modtage Cders Fjendes Datter , og unde hende Fristed og Tid til at forlige sig med sine egne , som forfolge hende , fordi hun elster den , der er Kjsd af Eders Kjsd og Ben af 3 ders Ben . Og indtil dette er naaet , vil jeg vandre fred- Iss om til > Vtraf , og forst vende tilbage , nåar I sige mig , at Eders <b>Hjcerter</b> ere stemte til Mildhed og Forsoning . Hvad jeg nu beder om , det er det , som ingen cerlig Riddersmand kan ncegte , om hans Had , end var not saa stort . " — „ Min <b>Moder</b> er Fromheden og Mildheden <b>selv</b> , " veddlev han , „ og hun har stor Magt over min Fader . Han tan vel vcere strceng og übsjelig , men han er en <b>sand</b> Riddersmand, cedel og hsjmodig , og end er ingen bleven skuffer , fom har ncermet sig ham med Tillid . Jeg haaber til Gud , at det stal lykkes mig at forsone ham , og bringe ham til at fole , at det er den bedste Hcevn , at forlade . Og da vil Eders Fader vel heller Me staa tilbage for ham , lhi saadanne Mcend knnne lige saa vel kappes i det gode , som i det stivsindede."

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008062004080'target=_>, 1861, Læsebog i Modersmaalet for Skolernes højere Klasser</a><h4>6625</h4>Tale ; thi paa rullende Hjul deres <b>Hjerte</b> blev stadt , og Ustadighed lagt i deres Bryst . " Agtelse for Alderdommen . „ Ad den graahaarede Taler aldrig du lee ! ofte , er godt hvad de Gamle tale ; ofte fra den skrumpne Belg komme fornuftige Ord . " Gjcestfrihed , Gavmildhed og Ve ' lgjorenhed mod de Fattige . „ Ild behsver den , som ind er kommen og er kold om sine Knee , Mad og Klceder har den Mand behov , som over Fjcelde har faret . " „ Vand behover den , som til Maaltid kommer , Haandklcede og gjcestfri Indbydelse , et velvilligt Sind , Ord og Gjensvar . " „ Med Vaaben og Klceder skulle Venner hinanden gloede . Givende og Gjengjceldende ere lcengst Venner , hvis Alt gaaer godt . " „ Udstjoeld ei Gjcesten og jag ham ei paa Dor ! Gjor vel mod de Fattige . " Klog Forsigtighed i Ord og Handling . „ Den varsomme Gjcest , som til Maaltid kommer , odsier ei med Ord ; lytter med Ore , stuer med <b>Oie</b> ; saa foler den Kloge sig for . " „ Fuldnok taler den , som aldrig tier med ufornuftige Ord ; fremfusende Tunge , hvis den ei tommes , tidt galer sig <b>selv</b> til Fortrced . " „ Fra sine Vaaben paa Marken man et Fodsbred ei vige ; thi uvist er at vide , nåar paa Veie ude Spydet behoves . " Mandehold ikke alene i Nydelsen af sandselige Goder , men ogsaa i Brugen af sin Magt . „ Vcerre Byrde paa Veien Ingen bcerer , end umaadelig Drik . " „ Glemsels Heire over Drikkelag svcever og stjceler Mcends Forstand . " „ Hjorden veed , naar hjem den stal , og gaaer da fra Grcesning ; men uklog Mand aldrig kjender sin Maves Maal . " „ Sin Magt hver klog Mand bruge til Maade ! Det han finder , naar blandt Tappre han kommer , at Ingen er raskest af Alle . " Tilfredshed og Munterhed . „ Glad og gavmild skulde hver Mand vcere indtil sit Livs Ende . " „ Ingen Sot er " vcerre , end den , med Intet at vcere tilfreds . " „ Hjemme glad bor Husbonden vcere , gjcestfri og klog . " Wrbarhed og Slebenhed i Omgang . „ Ei Mo stal Du lokke , eller Mands <b>Kone</b> , eller til Kaadhed dem egge . " „ Taaben gloer , naartil Gjcest han kommer , mumler eller hcenger ; facu ' r han sig en Drik , hans hele Sind er aabent . " „ Toet og mcet bor til Thing man ride , om man end ei er stadselig klcedt ; ved Sko og Brog bor Ingen sig stamme , ei heller ved sin Hest . " Omhu for at vinde sine Medmenneskers V elv il lie , iscer for at omgive sig med en fast Kreds af hengivne Frcender og troe Venner . „ Det Trce hentorrer , som i Torpet staaer , ei dcekker det Bark eller Bar ; saa er den Mand , som Ingen elsker , hvi stal han lcenge leve ? " Bautasteen staaer Veien ncer , med mindre Frcende den reiser efter Frcende . " „ Mod dine Frcendcr ulastelig vcer ! hevn dig ei , om de dig Uret gjore ; det siges de Dode at gavne . " Omhyggelig Behandling af den Dodes Lig . „ Dodes Lig du bjcerge , hvor paa Jord du dem finder , hvad enten de ere sotdsoe eller sodode eller vaabendode Mcend . Haug man opfore for den Hedenfarne , toe Hcender og Hoved , kjcemme og tone . for i Kiste han lcegges , og snste ham scel at sove . " gjcengelighed , og i Forbindelse hermed Strceben efter et godt Lov og efter en Bero mm el se , der kan strcekkc sig ud over dette Av og bevares iet <b>kjcerligt</b> og ceret Eftermcele . „ Saa er Rigdom , som Diets <b>Blik</b> , den er ustadigst af Venner . " „ Gods forgaaer , Frcender doe , <b>selv</b> doer man ogsaa ; men et hcedret Navn doer aldrig for den , som det erhvcrved'." „ Gods forgaaer , Frcender doe , <b>selv</b> doer man ogsaa ; men eet veed jeg , som aldrig doer : Dommen om enhver Dod . " Disse Leveregler have naturligviis vceret forskjellig opfattede og paa forskjellig Maade bragte i Udsvelse . Men i det Hele sinde vi dem at 22 *

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016101748020'target=_>Wildenhahn, August, 1859, Martin Luther</a><h4>82</h4>Margarcthe samkede sit stjonnc Hoved dybere end sædvanligt , og sank hen i Taushed . Ynglingen , dreven af en uscrdvanlig Hjertets Trang , traadtc nærmerc hen til den Blinde og loftede hendes Hoved sagte ivcirct . Dg da han saa , at hendes Kinder vare bedcekkede med en flygtig Rodmc , og at stille Taarcr brsde frem af de døde , omhyllede Dine , udraabtc han : „ Margarcthe , I stuffer mig ikke ! I har ogsaa eders Smerter , og over det himmelske Billede i Eder udbreder sig en Taage , der skjuler Gnds Herlighed for eders indre <b>Die</b> . Men saa <b>sandt</b> jeg tror paa en Forloscr , stal I blive befriet fra denne Smerte ! " „ Min Herre ! " for Pigen rystet op , „ hvilken Ret har I til at førstyrre mil <b>Hjertes</b> Fred ? " Derpaa vedblev hun strar mildere og næstcn ligesom bedende om Medlidenhed : „ Tal ikke videre , I tager feil ? Hvorledes kan I , som en Fremmed, vide , hvad Ingen har sagt Eder ! " „ Margarcthe ! " fortsatte Unglingcn , „ stal jeg sige Eder Navnet paa eders Smerte , eders Sorg ? I er Husets førskudte Datter ! det er Saaret , hvoraf eders <b>Hjerte</b> bloder . Men saa <b>sandt</b> Gud hjelper mig , skal dette Saar blive helbredet ! " Den Blinde rystede med et smerteligt Udtryk langsomt paa Hovedet , medens hede Taarer strommede ud af den mat indhyllede , dode Kilde . „ Gud er barmhjertig!" sagde hun derpaa stille hen for sig . „ Han har unddraget mig Vinenes Lys ; men han har dog ladet mig beholde Taarcrne . Hvor jeg dog er lykkelig , at jeg kan græde . I mange , mange Aar var ogsaa denne Naadegavc mig berovct . " Fichtncr vilde netop tale videre , da han Horte en Dor aabncs og lukkes i Sideværelset . Denne Bevcrgclse frembragte en rystende Virkning paa den Blinde . Ligesom opstrammet af en Drom for hun sammen og sagde : „ Herre , hvad sagde I nylig ? Tro mig , I tager feil ! Jeg har eders Ord fra igaar , at I intet Ondt har i Sinde mod os , dertil holder jeg mig ! Giv mig eders Haand paa , at I Ingen i dette Hus — horer I , Inge « vil gjore Ondt i Ord eller Gjcrninger ! " „ Margarcthe , " svarede Fichtncr alvorlig , „ hvor den rctfcrrdige og hcllige Gud er den talende Dommer , der kan Mennesket Intet gjore fra ellcr til . Mcn , og her har I min Haand derpaa , naar og hvor jeg taler eller handler her i Eders Hus , vil jeg knn være min Herres Tjener i Himmelen . Kan dette berolige Eder ? " „ Nei , nei ! " gjenmcclede den Blinde crngstcligt bedende og boldt ynglingens Haand endnu fast . Da gik Doren til Sideværelset op , og Fichtner havde Moie med at indtage en umistccnt ' elig Stilling . Det var Katharine , som traadtc ind . Hnn havde ifort sig en meget statselig Dragt , glimrende Ringe og Speendcr prangede paa Haand og Arm , en gron Silkehue med gylden Indfatning bedcrtkcde Issen og en Del af den fyldige , blonde Haarflette , paa hsirc Side hang i to Svlvtjcrdcr den sirlige , lille Taste , Symbolet paa den bydende og gestjcrftige <b>Husmoder</b> , og i hendes Dine saavclsom paa hendes fyldige , blomstrende Ansigt laa et Smil og den fuldkomneste Tilfredshed med sig <b>selv</b> . „ Tilgiv , Hr. Fichtner , " sagde hun i en yndig Tone , „ at jeg har ladet Eder vente ; „ cg værcr i min Faders Navn velkommen . " „ Det staar til mig , at bede Eder om Tilgivelse , " svarede Ynglingen . „ Jeg 2 *

<h4>224</h4>„ Paa Margarcthe hviler vort Huses Forbandelse ! " svarede Pigen trodsig . „ Ja , " tilfoiede Fichtner vemodig , „ ligesom Forbandelsen af det syndige Menneske, som offrer det uskyldige Dyr , hviler paa Offerlammet . Jomfru Katharinc , kan I virkelig fole Eder lettet , naar I har lagt eders Synder paa den stakkels Margarethe ? " „ Mine Synder ? " spurgte Pigcu med en stolt Mine . „ Ved I , hvad Ondt jeg har gjort ? Hvad Ondt kan I paavise mig ? Gud give , at eders Samvittighed var saa ren , som min ! " „ Arme , forblindede <b>Kvinde</b> ! " vedblev Ungliugen hcusyuslost . „ I bcrrcr rigtignok kun Kirkens og deus Prcrsters Brede ; I er ikke bleven lært noget Bedre , omcndskjont I her sidder ved den evangeliske <b>Sandheds</b> Kilde og kun behover at ose deraf . Men I vil ikke . , Skal jeg sige Eder , hvad der tynger paa eders Samvittighed ? I misbruger eders Faders svage <b>Hjerte</b> og hans førstyrrede Samvittigheds Uro til Fordel for eders hoffcrrdige Vcrsen og til Skade for eders Soster Margarcthe . I mener , at I elsker eders Fader ; men I skuffer Eder <b>selv</b> og ham ; I smigrer ham kun , hvor han i sin Daarstab og Svaghed skal befordre eders forfcrngclige Vcrsen . I mener , at I ogsaa er god og venlig mod eders blinde Soster ; men eders Godhed er hoffcrrdig Nedladenhed og eders Venlighed Bespottelse af sostcrlig <b>Kjcrrlighed</b> og Hengivenhed . I mener , at I gjor eders Pligt og Skyldighed ; men I ved cndnn ikke , hvad der egentlig er eders Skyldighed for Gud . I mener , at I ikke har nogen Synd , og eders <b>Hjerte</b> ligger bundet med hundrede Syndelcrnkcr , — I mener , I har Fred med Gild , naar I har bedet eders Noscnkrands og gjort eders Skriftcmaal og fnldbragt med Jammer og Sukke eders paalagtc Bodsgjcrning , — ak , I kjendcr aldeles ikke Freden med Gud , fordi I endnu ikke ved , hvor dybt I er falden . I er rolig og ved frit Mod , naar I har udsouct Eder med Kirken ! Arme , forblindede <b>Kvinde</b> ! Har I Magt til at tilintctgjorc eders Skyldbog for Gud , hvori Dommens Engel optegner alle eders Sosters Taarcr , som hun har grccdt for eders Hoffcrrdigheds , eders Herskesyges , eders Grnsomheds Skyld ? En eneste af disse Taarcr gjor al eders falske Fred til Skjcrndscl ! " „ Hor op , ti ! " raabtc Katharinc til ham i heftig Utaalmodighed . „ I taler om min Synd ? Ved I , hvori min Synd bcstaar ? Icg syndcr , idct jcg horer paa Eder , en fra Kirken , vor hellige <b>Moder</b> affalden Kjcrttcr , — idct jcg var svag og taabclig nok til at gaa i Forbon for Edcr hos min Fadcr , idct jcg , bcvcrgct af ufortjent Medlidenhed , vilde berede Edcr , som cn fremmed , hjemlos , af Kirken udstodt Mand et Tilflugtssted i min Faders Hus . Nu ser ogsaa jeg , hvad min Faders <b>Blik</b> strax opdagede hos Eder ; Logn er eders Vaaben og Bedrageri eders daglige Gjerning . I lyver med eders Navn og med eders Ansigt ! " „ Katharinc , " sagde Ynglingen vemodig , „ I er falden endnu dybere , end jez tænktc ! Gud forbanne sig over eders arme Sjcrl ! " „ Spar eders Medlidenhed for Eder <b>selv</b> ! " svarede Jomfruen trodsig og tri-

<h4>226</h4>umfercnde . I det samme ilede hun til Skabet , rev det voldsomt op , tog det lille Billede frem , holdt det for Unglingens Ansigt og sagde : „ Kjendcr I dette Billede ? " Neppe var Fichtners <b>Die</b> faldet paa Billedet , forend han greb efter det med begge Hænder , rev det ud af Jomfruens Haand , bedcrkkcde det med sine Kysse , brast i heftig Grand og sagde : „ Min <b>Moder</b> , min dyrebare , elskede , salige <b>Moder</b> ! " „ Altsaa virkelig ! " udbrod Katharine førskrækket . „ Den nlyksalige <b>Kvinde</b> , som har bragt Forbandelsen over vort Hus , er eders <b>Moder</b> ? " „ Forhaan ikte den forleste Sjcrl ! " sagde Fichtner med bydende Alvor „ Hvis Billedet af eders egen <b>Moder</b> ikke er ganske udstukket i eders <b>Hjerte</b> , saa ydmyg Eder for Ordet : Moderkærlighed . Katharine , " vedblev han i en mildere Tone , da han saa , at Pigen var truffen af hans Ord , — „ Katharine , jeg tor ikke for Eder lofte Sloret af en afskyelig Hemmelighed ; men det Ene maa I vide , at Guds <b>Oie</b> i dette Billede str Tyvens og Morderens Haand . " „ For alle Helliges Skyld , " bad nu Katharine crngstclig og førstrækket ved disse Ord , „ giv mig Billedet tilbage ! Hav Medlidenhed med mig ! Dersom min Fader kom og saa , hvad jeg har gjort , vilde hans Vrede blive strecktclig . Giv mig Billedet tilbage , og alt Ondt , som I har tilfoiet mig , stal være Eder tilgivet ! " Men Fichtner syntes ikke at hore , hvad hnn sagde . Hans <b>Blik</b> hvilede ligesom fortryllet paa den stjonne <b>Kvindes</b> Trcrk ; han kyssede gjentagne Gange Villedet, medens tillige hans Taarer faldt ned paa det . „ <b>Moder</b> , " sagde han stille hen for sig , „ hav Tak for din <b>Kjcrrlighed</b> , for din Sorg . Gud er mit Vidne , at jeg aldrig med Vidende og Villic har bedrevet Dig ! Jeg har elsket Dig af ganske <b>Hjerte</b> ; men jo mere jeg lærer at kjende den Herre , om hvem vi skulde fortalte Dig i din Dødsstund , og dog ikke <b>selv</b> tjendte Ham , — jo mere jeg ved , paa hvem jeg tror , — desto bedre ved jeg , at jeg ikke var værdig til din <b>Kjcrrlighed</b> . Bed til Gud for mig ! En Moders Bon kan den naadige Gud ikke afvise ! Kan et elendigt , fyndigt Menneske ikke bede for det andet ? " Derpaa satte han sig udmattet ned paa Lånestolen og trykkede Billedet snart til sin Mund , snart til sit Bryst , Katharine kunde dog ikke tillukke sit <b>Hjerte</b> for en saadan from , barnlig Kjcrrlighed. Hun saa med vemodig Deltagelse paa den unge Mand , der slet ikke syntes at ane , hvad hun led og stred for hans Skyld . Men da nn tunge Fodtrin hertes i Vcrrclsct ovenpaa , opstrcrtkcdes hun , ilede hen til Fichtner og sagde : „ Endnu engang for den hellige Marias Skyld , giv mig Billedet tilbage ! Jeg er dodsens , dersom Fader ser det i eders Haand ! " Men Fichtner svarede ikke ; hans hele indre Menneske hang ved Villedet . „ Min Herre , " vedblev nn Katharine med angstelig Paatrangcnhed , „ I har tilfoiet mig Smerte , I har mere end nogen Anden , min Fader ikke undtagen , med eders Ord gjcnncmboret mit Bryst ligesom med Dolkestik , — Alt stal være Eder tilgivet , giv mig blot Billedet tilbage . " „ Har jeg forvoldt Eder Smerte ? " spurgte Unglingcn , og en inderlig Glade straalede i hans Ansigt . „ Ak , Katharine , sig mig det endnu engang , at jeg har tilfoiet Eder Smerte , det vilde dog endelig være en lille Blomst paa et ode , ufrugtbart Land . Katharine , er det <b>sandt</b> , at mine Ord smerte Eder ? "

<h4>355</h4>„ Nei , " svarede den Blinde og ledsagede sit Svar med et smerteligt Smil . „ Aldrig , saalcrnge vi endnu bevcrge os i dette usle Liv ! Den samme dyrebare Johannes siger : Frygt er ikke i Rjcrrligheden ; men den fuldkomne'Rjcrrlighed driver Frygten ud . Men nn er jo Frygt og <b>Kjcrrlighed</b> vore Sjcrlcs egentlige Liv indtil Legemets Død , og er en faadan Hjertets Tilstand , hvor ufuldkommen den end er , dog tillige den hoiestc , som et menneskeligt <b>Hjerte</b> kan opnaa her paa Jorden . Mange have hverken Frygt for Gnd eller ' Rjcrrlighed til Gud ; mange Andre frygte vel Gud , meu de elske ham ikke ; knn den sande Christen frygter Gnd og elsker ham tillige . Men at elj ? e Gud uden Frygt , det kunne knn de kjcrre Engle og de salige Aandcr . " „ Margarethe , " udbrod Ynglingen , „ hvilket nyt Indblik i Guds Riges Hemmeligheder aabner I ikke for mig her ! Men lad mig fnldkommen ffne , hvad I ser . Gjor det , som I nu taler , tydeligere for mig . Jeg fer paa Eder , ligesom paa en Profctinde . " Margarethe lagde den venstre Haand over sine Vine , som om hun vilde tilhylle dem med et dobbelt Dcrkke , den anden Haand paa Hjertet og sagde : „ Kjcrre Herre , I maa , dersom I vil elske Gnd og prise ham , ikke ophoie Mennesket ! Dm det , som jeg siger Eder , er Indblik i Guds Riges Hemmeligheder , ved jeg ikke ; endog vor inderligste <b>Kjærlighed</b> til Gnd er nnderkastet Vildfarelser , fordi vor storstc <b>Kjcrrlighed</b> dog ofte ikke er andet end Svaghed og Daarskab ; men vil I kaste et <b>Blik</b> ind i Hemmelighederne af min Lro og min Rjærlighed til Gud , — nu , fan taler jeg gjerne med Eder ; thi I gjor jo mig <b>selv</b> lykkelig , naar I taler med mig om Gnd . " „ Ja det vil jeg , saalernge jeg aander og lever ! " ndraabtc Leonhard begejstret, tog Margarethes Haand i begge sine egne og trykkede den til sit <b>Hjerte</b> » „ Disse Vorn her med deres enfoldige og uskyldige Hjerter knnnc være Vidner til dette mit hoitidelige Lofte . " , / leg førstaar ikke Eder og eders Lofte , " svarede Margarethe , omendffjont hendes stcrrkt rodmende Kinder lod ane en Forstaaelse , som hnn vilde stjnlc . Men hun fattede sig hurtigt og fortsatte : „ Frygt og <b>Kjærlighed</b> cre de to Kilder til <b>sand</b> , christcli ' g Retskaffenhed , de vexlcnde Pulsslag i vort indre Sjcrleliv . Frygt og <b>Kjcrrlighed</b> cre det menneskelige <b>Hjertes</b> tvende Sider , af hvilke den ene gjor det Gode af Frygt for Gnds Vrede og Straf , den anden af <b>Kjcrrlighed</b> til Guds Godhed og Trofasthed . Frygt og Rjcrrlighed , snart hver for sig , snart forenede , ere de Trin , som føre til Gud . "

<h4>369</h4>ofte Timer , da jeg aner , hvorledes det vil være , naar Sjcrlens hele Liv ku » « er <b>Kjcrrlighed</b> til Gud ; men jeg har intet Ord dertil . Jeg kan kun sige : Jeg er salig i Haabct ! " „ Margarcthe , " begyndte hcr Unglingcn , medens en sl ' jcrlvende Tone gjennemtrcrngte hans Ord , „ Margarethe , jeg er det ogsaa og er det dobbelt ved Eder . Lad mig gjore Eder et Sporgsmaall Vi Seende sige , at Menneskets Sjcrl ligger aabcnbar i Ou ' cls Udtryk . Dit <b>Die</b> er rigtignok dodt og liulost ; men dit hele Ansigt siger mig , at Christns i <b>Sandhed</b> er dit Liv . Margarcthe , Dn kan ikke stue mig legemlig , men dit indre <b>Die</b> er vidnndcrlig klart . Sig , Margarcthe, hvorledes dommer Dn om mig ? " Pigen robede ved sine Miner og Udtryk synlig den storste Overraskelse og Forlegenhed . „ Hvorledes kan I falde pac » at gjore et saadant Sporgsmaal , kjcrre Herre ? " spurgte hun endelig . „ Jeg kan ikke lcrngcre berre det ! " vedblev Leonhard med trykket Stemme . „ Margarcthe , min Livsbane er forunderlig , Herrens Haand er hidtil alene i min Skjcrbne bleven saa synlig , at jeg ikke lcrngere kan ncrgte den , om jeg end vilde . Margarethe , jeg har Mod til at fremsige det : Gild vil , at vi ffnlle tilhore hinanden her og hisset . Margarethe , mit hele <b>Hjerte</b> ncrvner i <b>Kjcrrlighed</b> dit Navn ; vil Du , saa kan Du tilhore mig , saavclsom jeg Dig , i Liv og Død ? " Endnu forend Leonhard havde udtalt , var de » flygtige Nodme vaa Margarethes Kinder igjen førsvunden ; men over de snehvide Kinder banede klare Taarer sig Veien . „ Nor mig ikke nu , Leonhard ! " bad hun næstcn crngstclig , da hnn mærkcde , at Unglingen var traadt hende nærmcre . „ Jeg vidste det , forend I sagde det . Jeg beder Eder , lad mig være alene ! " „ Margarethe , " raabtc Leonhard i en smertelig Tone . ~ Nctop nu vil I forvise mig fra eders Nærhed ? " „ Det gjcrldcr ikke Eder ; men mig <b>selv</b> , " svarede hun med beklemt <b>Hjerte</b> . „ Guds Herlighed blænder mig , stakkels Pige . Ak , lad mig være alene , kjcrre Herre ! " „ Og jeg stal gaa udeu Svar , uden Haab ? " spurgte Leonhard . „ Margarcthe , ved Du ogsaa , at min forklarede Moders Velsignelse hviler paa mig og Dig ? Ved Du , at hendes Forfolgelse med Gud bliver fuldkommen i vor <b>Kjcrrlighed</b> ? " „ Jeg ved Alt ! " fvarede den Blinde . „ Jeg skulde for min Ringheds Skyld holde eders Ord for en Fristelse , om jeg har overvundet mit <b>Hjertes</b> Hoffcrrdighed. Men jeg kan det ikke . Jeg ved , at Herren vil have det saalcdcs ; men at jeg fsier mig i Herrens Villie nden Kamp , det er ikke rigtigt . Jeg kan ikke engang bede : Herre , din Villie ske ; thi det er min egen Villie , som Gud indgiver mig . " „ Saa vil Du tilhore mig , Margarcthe , for Tid og Evighed ? " spurgte Yng , lingen begejstret . Den Blinde lagde sine foldede Hændcr over begge Dine og sagde hoitideligt : „ Naar jeg for første Gang hviler ved min Faders <b>Hjerte</b> , og hans Forbandelse har forvandlet sig til Velsignelse , da vil jeg svare Eder , <b>selv</b> om I ikke sporgcr mig . Men lad mig nu alene . " Men Leonhard sagde rort : „ Margarethe , nu førstaar jeg Guds Ord og 6

<h4>468</h4>Hjerte . „ Tal om mig , Margcnetbe , tal med mig ! Sig mig isorvcicn , at . Du har tilgivet mig . " „ Dct kan jeg jo it ' kc ! " svarede Margarcthc rort . „ Jeg har aldrig anklaget Dig , hverken for mig <b>selv</b> eller for Gud . Jeg vidste , at jeg ikke var din Kjcrrlighrd værdig ; og naar dette bedrovede mig , saa var dct knn en Anklage over mig <b>selv</b> . Jeg førstod ikke at vinde din <b>Kjcrrlighed</b> . " „ O , stil Dig ikke saa hoit over mig ! " bad Kathaiine igjen . „ For har jeg vel trustet mig med saadan falsk Trost , men nn kunne saadannc Ord ikke incre berolige ung . Jeg ved , at jeg har foragtet Dig , jeg har ikke betragtet Dig som min t ' jcrrc Sostcr , mcn som cn lav Tjenestepige , jeg har ikke engang folt , at jeg har focacnsagct Dig Smerte ! " „ Og hvorfor bar Du gjort det ? " spurgte Margrethe med hjertelig Deltagelse . „ Ak , jeg ved det ueppc <b>selv</b> ! " svarede Kathariue . „ Saa langt jeg kan tamke tilbage i min Barndom , har jeg ikke sect dct anderledes ; Du har bestandig været Tjenestepigen og jeg Herskerinden ; og Du har baarct Alt saa stille og rolig . Du har ikte forlangt noget Bedre , Du har bestandig værct tilfreds og lykkelig . Ak , Margarcthc , havde vor salige <b>Moder</b> ikke forladt dette Liv saa tidlig , at jeg neppe kan mindes hende , saa havde dct ikke kommet saa vidt . Jeg tror , at Sostcrkjcrrlighed ogsaa maa lærcs . " „ Ja , ved den rette <b>Kjcrrlighed</b> til Gild ! " sagde Margarethe og vedblev derpaa venlig : „ Dette er det Eneste , kjære Soster , som har gjort mig bedrevet for Dig , nemlig at Du ikke har villet værr hjertelig elsket af Gud . Foler Du , Kathariue, at den kjccre Gud alene har fort Dig til mig ? " „ Endnu førstaar jeg ikke dette Ord ! " svarede Søsteren . „ Det kan nok være , at det er saalcdcs , men jeg førstaar dct ikke . Ak , jeg kan ikke skjule Noget for Dig ! " vedblev hun efter cn Pause og med fornyet Hjcrtcbetlemmelsc . „ En er det ialfald , som har fort mig til Eder , dct er den fremmede Svend . Margarethe , Lconhard var den Forste , som i mit Liv har sagt et uvenligt Ord til mig . Jeg har derfor bittert hadet ham og hadet Dig endnu mere end forhen , fordi jeg saa , at han havde Dig kjcrrcre , end mig . Jeg har villet forglemme hans skarpe Ord , mcn har ei kunnet det ; de vare ligesom Pidffcslag , der vilde helbrede mig , og pinte mig saalccnge , indtil jeg nu kommer til Dig . " „ Jeg ved ikke , hvad Leonhard har talt med Dig ! " sagde Margarethe . „ Men mener Du , at han har ment dct ondt med Dig ? " „ Indtil igaar har jeg staaet i den Formening , " svarede Søsteren . „ Men netop igaar , da jeg rakte.vor stakkels Fader hans Mad , kom han til , betragtede mig med en saa hjertelig og medlidende Vclvillie og sagde : Katharine , har Dn hoit om En , som oplukker den Blindes <b>Oine</b> og bringer de Stumme til at tale ? Disse Ord for mig gjenncm Marv og Ben . Jeg vidste , at han tænkte paa Dig og paa vor stakkels Fader , der endnu bestandig har værct stum og ikke bragt et Ord over stnc Lcrber ; men dog forekom dct mig , som om han vilde trcrffe mig , <b>selv</b> dermed , at jeg vel var blind med mine seende Mnc og at jeg skulde tale med Dig ! " „ Og derved tænkte Du paa Gud ? " spurgte Margarethe næsteu utaalmodig af Glcrde .

<h4>517</h4>Jammeren og en forfærdelig Advarsel om det mcnnestelige Legemes og Livs Skrobelighed og Forgængelighed . Den Syge foldede nu sine Hændcr og sagde : „ Min kjcrre Gud og Fader , har jeg undertiden været for letfcrrdig med mine Ord , saa ved Du jo , at jeg har gjort det forat fordrive mit svage Kjøds Vedrovclsc , ikke med ond Samvittighed ! " Og som der i denne Bckjendelse laa en ny Svaghed , hvorovc ' r han maatte stamme sig og anklage sig <b>selv</b> for Gud , vedblev han med al Kraftanstrcengclse hoit og lydeligt saaledes : „ I , Hr. Prcest , og I , Doktor Jonas og I Alle , som erc førsamlede om mig Docnde , jeg gentager det herved hoitideligt for Eder , I st ' nllc være mine Vidner til , at jeg Intet har tilbagekaldt af alt det , som jeg har skrevet mod Paven om Boden og Netfcrrdiggjorelscn , men jeg anser det Altsammen for det guddommelige Evangelium og for guddommelig <b>Sandhed</b> . Naar Nogle synes , at jeg har været for fri og for haard , saa angrer jeg det ikke . Jeg har jo ikke villet Nogen ilde , det ved Gud ! " Nu loftcde han atter sit <b>Blik</b> mod Himmelen og sagde : „ Endnu engang , Herre , dersom Du vil , at jeg stal do paa denne Seng , saa ste Din Villie . Ak , jeg havde heller villet udgyde mit Blod ! " Hede Taarcr vedbleve at vcrde hans Kinder , indtil et saligt Smil , der vidnede om Rolighed og Overgivelse i Guds Villie , udbredte sig over hele hans Ansigt. Han lukkede nn sine Mnc , lagde de foldede Hænder over sit Bryst , og kun hans Lcrber vedblev at bevcrge sig i stille Bon . En Dødsstilhed herskede i Vcrrrlfct , som Ingen vovede at førstyrre ; <b>selv</b> den stakkels Fru Katharine , hvis <b>Hjerte</b> vilde briste af Jammer , gjorde Vold paa sig <b>selv</b> og lagde alle sine Lidelser og Sorger ned i sin Mgtcfcrllcs inderlige Bon og Suk . Da udbrod Luther paa engang : „ Hvor er min lille Son ? Hvor er min allerkæreste Hans ? Lad mig se min kjcrre Son , at jeg kan velsigne ham forend jeg dor . " Og da nu en Pige havde bragt Barnet tilstede , tog Moderen det paa Armen og bar det hen til Faderens Seng . Da smilede Barnet og udstrakte længselsfuld sine smaa Hændcr til Faderen . Men denne formaaede ikke at lofte de tunge , iskolde Arme forat trykte Barnet til sit <b>Hjerte</b> . „ Nu , jeg kan dog se Dig ! " udbrod han med dcrmpct Stemme . „ Du gode , stakkels Var » ler ad mig , som ellers , ligesom intet Ondt var steet . Du fryder Dig , lig de kjcrre Engle i Himmelen , og ved ikke , at Du mcmste endnn idag stal blive et faderlost Barn . Mine Fiender og Modstandere ville ogsaa være dine ; de ville udstjcrlde dit crrlige Navn for min Skyld ; men jeg ved , at Gud er de Faderloses Fader og Enkers Trost og en Forsorger for dem Alle , og jeg vil ikke være bekymret for Dig . Herren velsigne Dig mit Barn ! " „ Jeg stakkels <b>Kvinde</b> ! " sagde Frn Katharine grædende . „ Hvorledes vilde det gaa mig , naar I skulde gaa bort fra denne Verden ! " „ Fat Mod ! " svarede Luther . „ Vil vor kjcrre Gud dennegang tage mig til sig , saa beder jeg Dig , at Du overgiver Dig i hans naadige Villie . Jeg ved vel , at Papisterne , som ikke knnne lilintetgjore mit Vcrrk , ville istedct derfor angribe

<h4>787</h4>som jeg i mit <b>Hjertes</b> Stilhed ikke vilde have fult for Dig . Da min Fader stedte Margarethc ud af Huset , da stjcrlvede jeg rigtignok af Medlidenhed med den stakkels Pige ; men da jeg tcrnt ' te paa , at dette Slag paa Margarcthes Hoved tillige maatte trisse Dig , da blev jeg igjen rolig . Ikke <b>sandt</b> , Leonhard , det er übesindigt af mig at aflcrgge et saadant Skriftemaal for Dig ? Men tro mig , at det er min sidste Glcrde i dette Liv , at jeg aflcrgger det , at jeg kan aflcrgge det . Jeg ved nu , at Herreu har ydmyget mig , og dcrpaa hviler mit Haab paa Guds- Naade . Leonhard , nu ved jeg , at jeg ci kunde reddes paa anden Maade , end alene ved den fuldkomneste Udmygelse af mit hovmodige <b>Hjerte</b> ! " „ Katharine , " sagde Unglingen , „ hvis jeg har gjort Dig ondt med mine Ord , saa var det fordi jeg var hjertelig bedrevet over dit indre Menneskes Tilstand . Jeg har aldrig hadet Dig , jeg havde for din Skyld kunnet bringe ethvert Offer for at førskaffe Dig en <b>sand</b> Lykke og . Fred . Tro mig , Katharine , jeg straffede og tugtede Dig , fordi jeg hjertelig elskede Dig ! " „ Det er godt , at Du først nu siger mig det , Leonhard ! " svarede den Syge med et flygtigt vemodigt Smil . „ Tidligere havde jeg aldrig knnnet tro Dig , eller i min taabclige Daarst ' ab førstaact Dig . Kald Margarethc hid til mig ! " udbrod hun nu pludselig . „ Ilden i min <b>Kjcrrlighed</b> stammer igjen op , og dennegang vil den ombringe mig . Det forekommer mig , som om vanvittige Tanker ville trcrnge ind i mit Hoved ; hvad jeg endnu vil gjore , man jeg gjore nu strar . Er Margarethc tilstede ? " Fichtncr havde fort Margarethc hen til hende og stod med hende ved den Syges Seng . „ Træder ucrrmcrc ! " bad Katharine . „ Jeg onskcde at velsigne Eder ; men jeg kan ikke oploftc mine Hænder . Lcrggcr eders Hændcr paa mine ! " Og da de To havde gjort dette , vedblev hnn : „ Margarcthe , af din Mund har jeg første Gang hort det Skriftens Ord : Hvo sig <b>selv</b> ophoicr , skal fornedres, og hvo sig <b>selv</b> fornedrer , skal ophoics . Se nn , kjcrrc Soster , Herren har gjort sit Ord <b>sandt</b> . Han har kastet mig fra min forfcrngelige Hoffcrrdigheds Hoide ned i Stovct , og ophsict Dig af din Lavhed . Havde jeg forudscet dette i min Blindheds Dage , saa havde jeg dadlet Guds Styrelse ; men nu ligger jeg i Stovet paa mine Kncr og beder og takker , at Herren i Himmelen er en retfærdig Dominer og en naadig , barmhjertig Gud . Margarcthe , ligesom jeg engang foragtelig saa ned paa Dig , saaledcs ser jeg nu ydmyget op til Dig . Jeg ansaa Dig for vort Hus ' s Forbandelse , og se . Du var Velsignelsen for vort Hus , Guds Naadcgave , som skulde redde mig og endnu vil rcdde min ulykkelige Fader . Al Livets Lykke , som jeg i mit Livs bedre Timer har dromt , haabet og nedbedet for mig fra det Hoie , den være din i overflodigt Maal . Min Vods Taarer gjor Herren til en himmelsk Glcrdeshost for Dig . Du har tjent mig som en simpel Tjenestepige her paa Jorden ; jeg vil hist i Herrens Rige , hvis Han ellers endnuvil modtage mig , være din Tjener . Veer en lykkelig <b>Kone</b> med denne Mand , den Eneste paa Jorden , for hvem mit <b>Hjerte</b> skrukkedes og ydmygede sig . Kysser mig , dersom I ikke styr Berorelse med en Doende , kysser mig , jeg kan det ikke ; kysser mig til Tegn paa , at I cre førsonede med mig ! " 13 *

<h4>863</h4>Sengen og indtog en saadan Stilling , at han kunde se den Syge lige i Dincnc . „ Sygdom er en hciard Byrde ; den , som ikke maa bære den , barrer den ikke . Hvad feiler Eder , kjcrre Mester ? Har I mange Smerter , eller er eders Sygdom mere end Legemssvaghed ? " „ Svaghed ! Svaghed ! " svarede Hombcrgcr modvillig . „ Og hvor lcrnge har I folt denne Svaghed ? " spurgte Luther videre . „ Jeg mener , " vedblev han venligt , da den Syge saa paa ham , som om han ikke vidste , hvad han skulde sige , eller som om han aldeles ikke havde førstaact Sporgsmaalet : „ Jeg mener , om det allerede er lcrnge , siden I begyndte at fole Eder saa legcmssvag og skrobclig og maattc gaa tilsengs ? " „ Lcrnge , lcrnge ! " svarede Hombergcr , som hidtil ; men tilfoicde dog strax efter : „ Jeg har værct syg lcrngcre , end jeg har værct sengeliggende ; thi mig har stor Ulykke truffet ; jeg har mistet mit kjcrre Barn , og min <b>Kone</b> har jeg ogsaa mistet , og ellers har det gaaet mig overmande sorgcligt i Verden . Jeg var endnu en Gut van ti Aar , da min Fader og <b>Moder</b> dode ; jeg har fra Barndommen af maattct lide meget Ondt , og der gives vist ikke mange Mennesker i Verden , hvilke have lidt saa meget som jeg . " ' „ Og hvorledes har I da baarct alt Dette ? " spurgte Luther i stille Glcrde over , at den Syge nu begyndte at tale . „ Hvorledes jeg har baaret det ? " udbrod Homberger . „ Jeg har stedse levet retskaffent , og jeg onskede at se den , som kan have noget Ondt at udscrtte paa mig . Jeg førstaar min Kunst , og jeg kan med Dokumenter bevise for Enhver , som onst ' cr at vide det , at jeg har arbeidet redeligt . Jeg er ingen gjerrig Mand ; jeg har betjent mangen fattig Landsbykirke for Intet og hjulpet mangen fattig Svend « Der kom engang En til mig , — det er nu vel femten Aar siden ; ham hjalp jeg med Klcrder , Sko , Penge og Gods , og jeg har endnu idag det Brev , som han skrev til mig fra Nnmberg , og hvori han takker for al <b>Kjcrrlighed</b> og Godhed , som jeg har bevist ham . Dog , I ffal <b>selv</b> lcrse Brevet ! " . „ Min kjcrre Mester ! " svarede Luther , da han saa den Syge reise sig i Sengen og gribe efter et Skab i Vccggcn . „ Hvad ffal det nytte til ! Jeg tvivler ikke paa <b>Sandheden</b> af det , som Inu beretter mig . Men det er hellerikke det , jeg onskede at vide af Eder . I har sagt mig , at I har havt ret mange sorgelige og onde Dage i eders Liv , og da spurgte jeg Eder , hvorledes I har baarct alt dette ; stille og taalmodigt , eller med modvilligt <b>Hjerte</b> ? " „ Hvad mener I med de onde Dage ? " spurgte Hombergcr hurtigt og uvenligt . „ Onde Dage , " svarede Luther i al Rolighed , „ ere saadannc Dage , som komme med alslags legemlig og ciandelig Nod og Anfcrgtclse , saaat vi tro , at vi ci t ' nnnc holde det ild og ville fortvivle ; og onde Dage kaldes de vel ogsaa af den Grund , at de efterlade » neget Ondt og drive til Ondt ! " „ Jeg har erfaret meget i mit Liv ! " sagde Hombcrgcr undvigende , og med et mort ' t , uvilligt <b>Blik</b> paa Luther . „ Og har I baarct det til Velsignelse for Eder ? " fpurgtc Denne . „ Ja , ja ! " førsikrede han . „ Det er vistnok bleget mig snrt ; men Gud har bestandig velsignet min Kunst . Jeg er vistnok nn en fattig Mand ; men Næring , 13 *

<h4>879</h4>strint mod mig . Jeg vidste , at hun havde mig kjeercrc end Pinetta ; var det nu cn Synd , at jeg glcrdede mig over hendes Gunst og vilde vinde hende for mig ? " „ Mester Hombcrgcr ! " udbrod Luther alvorligt . „ I ved ikke , hvorledes I med saadannc . Ord anklager Jomfruen i Mailand . I vil vaske Eder ren for den Synd , som I har bekjendt Eder skyldig i , og leegge den paa <b>Kvinden</b> , ligesom om <b>Kvinden</b> havde forfort Eder . Var det vir Ulig faaledcs , som I siger , faa gjor I Jomfruen til en gemen Tos , der bryder Troskaben mod sin Brudgom, og som har gjort det alvorligste Skridt i sit Liv med letsindigt Gemyt , idet hun har forlovet sig med cn Mand , hvem hun ci kunde elske , driver cn , ugudelig Spot med sine Forcrldrc og Guds Ord , og fryder sig over sin Brudgoms Kvaler . Mester Hombcrgcr , " vedblcv Luther mere og mere alvorligt , „ I ved ikte , hvorledes I vanærer Eder <b>selv</b> med et saadant Ord . Den , som kan modtage en <b>Kvindes</b> Gnnst , som tilhører en Anden , har intet crrligt Sind for sig <b>selv</b> , ved ikke hvad tro , crrlig <b>Kjcrrlighed</b> er , begjærer for sig <b>selv</b> hellcr ikke nogen tro <b>Kjcrrlighed</b> og , maa allerede i Forveien have forudseet , at <b>Kvindens</b> letsindige , vaklende <b>Hjerte</b> snart vender sig til cn Trcdie og Fjerde . Har I bctænkt alt dette . Mester Homberger ? Var den mailandskc Jomfru virkelig en saadan Tos , som idag forlover sig med en Mand , og imorgen ser sig om efter en anden ? " Hombcrgcr forblev nogen Tid i Tanshed , hvori han synlig pintes af modstridende Tanker . Endelig fagde han med et neesten bebrejdende <b>Blik</b> paaLnther : „ Folk have Ret ! Eders Ord erc haarde og spidse og sonderskjcrre Sjcrlen . I har Vclbchag i at gjore Mcnncskct til Synder ; I gleeder Eder over cn førstyrret Samvittigheds Kval . " „ Stakkels Ven ! " udbrod Luther med deu hjerteligste Deltagelse . „ Jeg har ogsaa erfaret , hvad en førstyrret Samvittigheds Kval er , og kjender nforsonede Synders Byrde ; men da jeg ogsaa ved . Gud veerc takket , hvilket fornoiet Liv og hvilken Salighed der er , hvor Syndernes Forladelse er , saa velsigner jeg denne Kval , fordi den tilsidst i Gnds Naade og Barmbjertighed bliver til salig Fred . Men hvorlidct kjender ikke I mig og det Ord , paa hvilket jeg staar . Det Antal af mine Fiender og Modstandere , som gjerne onskede at bringe mig til Tanshed og Undergang , er stort og har megen Magt ; og hvormcgen Smerte de allerede Have tilfoict mig og med hvilken Pine de have plaget mig , kan neppe beskrives . Men det maa I tro mig , at jeg heller ikke onst ' cr min storste og giftigste'Modstandcr den Kval , som tynger paa et af Synd sonderrcvet , med Gud endnu uforsonet Gemyt . Hvor Englene endnu sorge og græde , der kan en retskaffen Christen aldrig , fryde sig . Men lad mig sige Eder , kjerre Mester , hvad der ikke behager mig hos Eder . Jeg straffer i Eder paa Gnds Ords Grund deu Gjerning som . en S ) ? ttd , hvilken I maast ' e gjenncm lange Acir har betragtet som cn ugyldig Gjerning . Derfor er I vred paa mig , ligesom om jeg frydede mig over eders Kval , og jeg vil dog intet Andet , end eders <b>Hjertes</b> Fred . Svar mig , " vedblev han fortroligt , „ jeg vil gjore Eder nogle Sporgsmaal . Har den mailandst ' e Jomfru med tydelige og klare Ord sagt Eder , at hun havde Eder kjcrrcre , end sin . Brudgom ? "

<h4>1007</h4>at jeg nu snart stal vare der , hvor min Frelser er , — men den kjære Gud vil tilgive en docnde <b>Moder</b> det , om Langsclen efter hendes Barn er ovcrvciende . Thi De maa vide , Hr. Pastor , at den kjeere Gud gav mig et ret elskvardigt Varn ; det var min hele Rigdom paa Jorden ; al min <b>Kjcrrlighed</b> , mit Haab , jeg <b>selv</b> var liundet ved dette Barn . Dette var syndigt , — derfor tog Gud fra mig dette Barn ; men han tgger nu Moderen ogsaa , og det førsoner mig igjen med ham . Ikke <b>sandt</b> , Wilhelm , Du er ikke vred paa mig , fordi jeg forlader Dig ? " Hun rakte fin Mand paany Haanden , og da denne grad hoit , sagde hun : „ Ak , grædcr Du allerede igjen ? — Du ved , hvor ondt det gjor mig , — jeg kan jo ikte grade « led . — Ja , Hr. Pastor , i tre Maaneder bar jeg amaabt Gud om en eneste Taare ; — men Alt er fortorret i mine Dine . Wilhelm , bring mig dog min Ligkrands . " — Manden bragte en af dode Blomster og med mangfoldig Guld- og Solvstads prydet Krone . — Jeg for op : „ Hvem har gjort denne ? " „ Jeg <b>selv</b> , jeg ganske alene , — sagde den Syge , — i mange Uger har jeg arbeidet paa den hver Dag , og imorges indsyede jeg den sidste Blomst . Det var netop paatide ; thi Arbeidet vilde siet ikke mere gaa fra Haanden . Derfor tror jeg ogsaa , at min sidste Time er kommen , " — Med disse Ord tog hun Kronen i Haanden , foldede og stellede paa den , som en <b>Kvinde</b> pleier gjore med sin verdslige Pynt , lod mig dernæst bese den fra alle Sider og spurgte : „ Ikke <b>sandt</b> , Hr. Pastor , — denne Krone behager ogsaa Dem ? " „ Forfamgclighed cndnn i Døden , " — tænkte jeg . Hun maatte vist gjcrtte mine Tanker og sagde : „ Men jeg skal ikke bare den paa Hovedet , — den skal blot som mit sidste Arbeide gaa med mig til Graven . Og dog , — at jeg kan sige Dem Alt : I Bibelen staar skrevet : Var tro indtil Døden , saa vil jeg give Dig Livsens Krone . Da jeg nu , — Gud er mit Vidne dertil , — ikke er nogen Trolos , saa vilde Ordet Krone heller ikke gaa nd af mit Sind , og jeg havde ingen No , forinden jeg fik denne Krone fardig . " Jeg ved vel , at Vibelens Ord erc at forståa om en anden Troskab , end <b>Konen</b> forklarede sig den ; men jeg havde ikke Mod til at snare hende . Medens hun talte saaledes , kom en ung , omtrent 2 Naarig Pige ind i Vcrrclsct og saa med synlig Rorelse hen paa den Syge . „ Bchover Du maaske Noget , Marie ? " — spurgte hun . Den Syge rystede smilende paa Hovedet og sagde : „ Intet , siet Intet . — Jeg er fortorret i mit hele Legeme , — vedblev hun henvendt til mig , — jeg forekommer mig <b>selv</b> som en Billedstottc af Marmor ; jeg sulter ikke , jeg torster ikke ; jeg er bestandig trat og kan dog aldrig sove . Ikte engang mine Vine kan jeg tillukke , det forekommer ung , som om jeg derved gjor mig ondt . Saaledes er vel Guds Vie bestandig aabent , — og dermed troster jeg mig , naar jeg om Natten , efterat min Wilhelm <b>selv</b> eller min kjeere Soster ere indsovede aflang Vaagen, er igjen ganske alene og med utrattelige Vine betragter Skjcrrmcn foran Nattelyset. Ja , det forekommer mig uudcrtiden , som om jeg da saa ind i Guds eget <b>Die</b> , og som om det <b>Die</b> sagde til mig : „ Hold modigt ud ; Du ffal snart vare forloft . "

<h4>1027</h4>Jeg gjorde den Syge opmærksom paa , at der med Nydelsen af den hellige Nadverc , hvorefter ' hendes <b>Hjerte</b> dog lcrngedcs , ogsaa var forbuudct en Forpligtelse til christclig <b>Kjcrrlighed</b> og Tilgivelse af alt det Onde , som er tilfoict os af andre Mennesker , — at jeg saaledes inaatte ause hendes <b>Hjerte</b> som utilstrækkelig renset , aldeustuud hnu ci havde bortfjeruct fra det enhver Uvillie og Vrede . Derpaa svarede hun mig , at hun aldeles ikke nærede Uvillie eller Vrede mod Nogen , og af Hjertet tilgav alle siue Ficndrr ; men at hun ikke mere ventede nogen Hjcrlp af Menneskene , just ikke fordi de ci kunde , men fordi de ci vilde . Jeg erfarede først senere , hvad huu vilde sige dermed . Hun havde nemlig paa den hele Guds Jord ingen Slcrgtninge , undtage » c » gift Soster , der boede ctpar Mile derfra ; meu som i de to Aar , i hvilken Tid den Syge havde været sengeliggende af Tcrring , kun en cucste Gang , og det efter en barsk , nwstcn voldsom Opfordring , havde besogt hende i nogle Minutter . Og det just ikke af Sestcrhad , — men af Ligegyldighed , af Folcsloshed , — af Bekvemmelighed og Fornoiclscssygc , der ikke gjerne dvcrler hos den Syge , netop fordi der sammesteds ei gives nogen sandsclig Fornoiclse . Imidlertid var en Nabokone traadt ind i Vcrrelset , — den eneste menneskelige Sjcrl , som bekymrede sig om den Syge , rengjorde og opredte hendes Seng og bragte hende en Suppe . Jeg talte uu med deu Syge om det Trostende i Nydelsen af den hellige Nadverc paa Sygesengen , sogte saamegct som mnligt at lede hendes Tanker og Folclser ene og alene hen paa det Himmelske , og spnrgte hende for Skriftcmaalct , om hun maast ' e havde noget paa Hjertet , som hun onskede at meddele Nogen . Huu smilede smerteligt , — saa paa mig med vemodsfulde Niue , rakte mig Haanden og sagde : „ Jeg knnde fortcrlle Meget , — men jeg har ikke Kraft dertil . Gud har i denne <b>Kone</b> sendt mig en god Engel , hnn kjender min hele Smerte , — hnn vil fortcrlle Dem den . Blot saamegct siger jeg Dem : Jeg anklager Ingen . Christus , min Frelser , har befalet mig : Velsigner dem , som eder forbande , og beder for dem , som gjore eder Skade . " — Da jeg nu skriftede i de » Syges Navn , foldede huu sine Hændcr og scrukede Hovedet ned paa Brystet , saaat de sorte Håar igjen bedcrkkcde det hele Ansigt ; hnn cftcrsagde borligt hvcrt Ord og saa dcrnæst paa mig med en vemodig Gleede , da jeg lovede hende Haab om hendes Synders Tilgivelse . Hun nod den hellige Nadverc med en Frimodighed , hvilken jeg onstede enhver Christen havde seet , som hos sig <b>selv</b> endnu ikke har folt <b>Sandheden</b> af den Scrlning : „ I G , ud leve , rorcs og cre vi , " , og „ haus Barmhjertighed har ingen Ende og haus Trofasthed er stor . " Deu Syge modtog med stor Glcrde mit Lofte at besoge hende Dagen efter , og lcruede sig trert og tilsyneladende ganske udmattet tilbage paa Pude » . Men Naboersken gik ud med mig og fortalte mig Følgc « dc : „ Den Syge er c » ung <b>Kone</b> paa 33 Aar . Hun levede i et lykkeligt Wgte-

<h4>1120</h4>nemsigtige . Den ene kostelige Perle er ogsaa orientalff , fra det hellige Osterland , fra Oliebjergets Fod , fra det galileriske Haus Bred . Der levede og døde engang En , som havde bragt denne kostelige Perle med til os fra det evige Morgenland , til hvilket han igjen er vendt tilbage . Dens klare og gjenncmsiqtige Glands straalcde over Bjerg og Hav , gjcnnem Neettcr af Vildfarelser og Synd og lyse atter i de Troendes Hjerter . Kjobmanden solgte Alt forat kjøbe den ene kostelige Perle . Ak , hvor mange Mennesker give ikke denne Perle til Pris for andre , falske , stette Perler ! — Men hvad betyder da denne Perle ? — Det er vor Christendom , vor barnlige Tillid til Gud , som berrer os Alle paa sit Faderhjerte , som forer os gjcnnem Lidelsens Nat til Livets klare Morgen . Det er vor <b>Kjærlighed</b> til ham . Menneskenes Frelser , Jesus Christus , der er bleven os Veien til Faderen , den evige <b>Sandhed</b> , det egentlige Liv , som med Suskcudekjeerlighedens Baand forener os med alle Jordens Mennesker til en stor Familie . Det er vort Haab paa et christeligt Liv , paa en Retfærdighedens Dag , paa et Gjensyn med vore Kjcrre . Dette er den ene kostelige Perle , som ci kan tjubes med alt Guld paa Jorden, og som vi ofte nok tabe for vore Lidelsers og Smerters Skyld . Menneskene beerc vel mange Perler som Beskyttelsesmidler for Onder og Vcsværinger ; deinre ene Perle skulde de bestandig beere paa deres <b>Hjerte</b> . Er der Uro og Storm derinde , saa anlceg blot Perlen , den vil fordrive Uvcirsnattcn og lade den klare Himmel igjen opgaa . Og naar I engang do og Dagen her paa Jorden med sit Solskin forvandler sig for eders <b>Oine</b> til Morte , saa vil den ene Perle blive for Eder en gylden Stjerne , der viser hen til det evige Morgenland, til vort fcrllcs Fædreland . Kjcrre <b>Kone</b> , — hvor lykkelig er ikte De , — at denne ene Perle ogsaa er deres Eicndom ! " —

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016052528005'target=_>Harboe, Ludvig, 1857, Psalmebog til Guds Ære og Hans Menigheds Opbyggelse</a><h4>889</h4>Med min Nand og svage Sind , Udi Morthed viklet ind , Saadan Naadcs Dub ndgrnnde ! Alt tilsidst en Ende faaer , Men Gnds <b>Kjærlighed</b> bestaaer . 4. Han sin Aand inig gi ' er til Eie I sit Ord , ved hvilken han Forer paa > c trange Veie Mig til det sorja ' tted ' Land ; Aanden og mit <b>Hjerte</b> fryder Med en <b>sand</b> og salig Tro , Som førstyrer Satans Ro , Ja <b>selv</b> Dødens Magt nedbryder ; Alt tilsidst en Ende faaer , Me » Gnds <b>Kjærlighed</b> bestaaer . og Hjæly i Nodens Tidi Naar min Kraft , min Magt , min Styrke Intet mccgter , Intet fan , Tager Gnd mig ved min Haand , Og begynder <b>selv</b> at virke ; Alt tilsidst en Ende faaer , Men Gnds Kjanligbed bcstaaer . 6. Jorden med alt hvad den eicr , Himlen med dens ganske Hær , Hvor , » ! t <b>Oie</b> sig hendreier , AlttilGo « Fjelde , Mark og Ferske Vand ' nied Havets Ginuo S ! > i » c ra ' kke > nig mi » Fode ; At tilsidst en Ende faaer Men Gnds Kærlighed bcstaaer . ? . Naar jeg sover , Herren holder Vaagct <b>Oic</b> over mig , Og ved hver Dags Morgen volder , At hans Gunst fornyer sig ; Havde Gud ei taget vare , Og mig trolig staaet bi . Var jeg ikke sluppet fri Af saa megen Nod og Fare ; Att tilsidst en Ende faaer , Men Gnds <b>Kjærlighed</b> bestaaer . 8. Hvad har Satan dog udgrundet Mange Plager , mangen Straf ; Jeg har dog til Intet fundet , Herren har det værget af ; Ja den Engel , Gnd mig sendte , Sjælefiendens Skalke-Naad Og " besluttet onde Daad Ganske langt fra mig bortvendte ; Alt tilsidst en Ende faaer . Men Gnds <b>Kjærlighed</b> bcstaaer . 9. Som en Faders <b>Hjerte</b> ikke Slet tillukker Randens Dsr for sit Nnrn som ei vil Me Sig saa ly « digt . som det bsr ; Saa min fromme Gud og Fader Ojsr mod mig yaa samme Viis , Straffer mine Feil med Niis , Og sit Sværd ei rande lader ; Alt tilsidst en Gnde faaer , Men Guds <b>Kjærlighed</b> bestaaer . M Omendssjsndt den Straf og Plage , Som afGud mig lcrgges yaa , Noget bitter knnde smage , Ber jeg

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014011608145'target=_>James, John Angell, 1868, unge Qvinde</a><h4>263</h4>Eller skulle vi fra denne enkelte Virkekreds kaste vort <b>Blik</b> paa det almindelige Gode , som hun ved sin Indflydelse udMer paa Folket ? Hvo ved itte , at Kilden til et Lands Velfcerd er at indenfor den huslige Kreds , i lykkelige Familieforhold . Vlot en eneste faadan Familie er et Vidrag til dets Magt og Velstand . Knnne saadanne Familier beståa uden Qvindens Wgt , Omsorg og Ledelse ? Er det ei blevet et Ordsprog , at Hjemmet altid har vceret Planteskolen for store Mcend og Modrene deres Lærerinder ? Man kan vove at paastaa som en almindelig <b>Sandhed</b> , at <b>Qvinden</b> ikke alene er sit Varns legemlige <b>Moder</b> , men ogsaa dets aandelige . Hende er ' den forske Vearbeidelfe af dets <b>Hjerte</b> og Sjoel anbctroct . Tanke , Fcilelse , Indbildningskraft , Dyd , Religion eller ogsaa Modsætningen dertil , Alt vorcr under hendes opdragende Indflydelse . Den storste moralske Magt er den , en <b>Qvinde</b> udover over sine Vorn . Tet afgjMende i Opdragelfcn er just Begyndelsespunktet <b>selv</b> . Den herskende Retning , som skal bestemme hele Livets LM , ligger allerede skjult i de ftrste Varndomsaar ; men disse tilho ' re Moderen .

<h4>429</h4>Opftrsel . Skulde din Sjcel gaa fortabt , saa vilde det ikke vcere nogen Undskyldning for Guds Domstol eller nogen TrD for dig <b>selv</b> i Fortvivlelsens Verden , at du lod dig indtage af Nvillie mod Gudsfrygten ved nogle af dens Vekjenderes urigtige Opforfel . Og nu , ved - Afslutningen af dette Capitel , lader mig opfordre Eder , I Unge , til allerede ved Livets Begyndelse at betcenke det store Maal , for hvilket I af Skaberen sattes ind i Verden . Tomter ikke altfor ho ' it om Eder <b>selv</b> , thi I ere Syndere , I saavel som Alle . I behøve ogsaa I , men I kunne ogsaa vinde den Frelse , som er beredt i lesns Christus . Tomter ikke heller altfor ringe om Eder , thi I ere udødelige Vcesener og kunne arve et evigt Liv . Hcover Eder til Eders sande Vcerdighed ved at hceve Eder til en <b>sand</b> Gudsfrygt , Tro og et Liv i Saudhedcn . Forspilder ikke Eders Liv i smaaligt , om ogsaa uskyldigt Tidsfordriv, uvcerdigt Eders Evner , Eders Vestemmelfe og Eders Pligt . Betragter med klart <b>Blik</b> og fuld Bevidsthed Eders Kald for denne Verden og Eders Vestemmelfe for den tilkommende. Ser , hvorledes et Kald her aabner sig for Eder , hvilket I maa opfylde med en Christens Hellighed , Mre og Velsignelse saavel som med alle en svindes Dyder og Fortrin. Drager Eders <b>Hjerte</b> fra Forfængeligheden , skjenker det til Gudsfrygten . Skjenker Gud Eders Livs Morgen , og da stal dets Dag , den blive lang eller kort , klar eller overskyet , ensom eller tilbragt i Forening med Andre , lys eller mprk , rolig eller stormende , om den end ender ved en bastig Hjemscigelse , saadan som Eders KjM vel kjender , eller om den gaar ned i en lang og mork Aften , den stal dog for Eders salige Aand , befriet fra StMets Byrde , afluses af den skyfrie og evige Morgen , fom ikke efterfølges crf nogen Nat . Da skulle I finde , at <b>Kvindens</b> saavel som Mandens Maal var at forherlige Gud og nyde af hans Vcesen for evig .

<h4>602</h4>Ore , den Lammes Fod og bringe den Sorgfuldes <b>Hjerte</b> til at synge med Fryd . Jeg vender nu tilbage til den Tanke , hvormed jeg begyndte , at en Datters rette Opftrsel i Hjemmet er at ftge at gjore Hjemmet lykkeligt . Der ligger et Trylleri i selve dette Udtryk , et lykkeligt Hjem . Og hvad det angaar , som kan kaldes det hjemlige Livs Poesi , kan der vel findes en elskeligere Blomst i denne det jordiske Paradises Have end en god Datter og hengiven Spster , prydende sin jomfruelige Unde med de Dyder , som anstaa hendes KjM , hendes Alder og Stand , samt forhalende og forædlende alle sine Mrige fortræffelige Egenskaber ved den Tro og <b>Kjcerlighed</b> , som gjor hende elskelig ogsaa i Guds <b>Oine</b> ? Sin Faders Stolthed , sin Moders Gloede , sine Brødres Selskab er hun for dem Alle en god Engel . Hvor megen Gloede bereder hun ei dem Alle , hvor megen Lykke udstror ikte hun , den foelles Elskede , blandt dem ! Hendes Fraværelse beklages som et almindeligt Tab , hendes Tilbagekomst til Familiekredsen hilses som Gjenerhvcrvelsen af en tabt Gloede . Og naar dette elskelige Boesen engang bliver elsket af en Anden , der ikke tilhorer Familien , omendskjMt ved hende ncervcd senere at forenes med den , med hvilken Skat kommer han ikke da , skjont pa « deres Bekostning , til at pryde sit eget Hjem ! Deres <b>Hjerte</b> blader ved Tanken om at skilles fra hende af en Smerte , som Intet uden Haabet om hendes Lykke kan lindre . Hendes Bortgang efterlader en Tomhed , som endog Tanken om eller Synet af hendes neppe kan adfprede . Men fordi Qvindens Kald og Maal er at lyksaliggjore sin Wgtefcellcs Hjem , faa lader os antage , at hun er gaact for at opfylde det . Hun er bleven godt opdraget, hun har ogsaa opdraget sig <b>selv</b> godt i Forældrehjemmet for at kunne lyksaliggøre sit eget , og alle den gode Datters og den gode Spfters gode Egenskaber forenede skulle nu udvikles og gaa op i de mere modne , <b>husmoderlige</b> Dyder , bos Wgtefcellen , Moderen og Husmoderen. Og hun , fom saa vcesentlig bidrog til LyMen i Forældrehjemmet, har paa den cedleste og bedste Maade forberedt sig til i endnu hsiere Grad at bidrage til det egne

<h4>917</h4>kan indgive . Nu er Kærligheden den kraftigste Bevæggrund i og for Menneskenes Handlinger . „ Med Mcnncskcbaand trak jeg dem , " sagde Gud , „ med <b>Kjcerligheds</b> Reb . " Heri ligger den sande Hemmelighed baade af Menneskets naturlige Sjeelccnltur og af Evangeliets himmelske Magt . Menneskenaturen er snarere dannet til at styres og ledes af Kjeorlighcd , at drages med Kjcerlighcdeus og ømhedens Vaand , end til at slwbes fremad i Strenghedens Kjceder . OgQviudebjcrtet er skabt til at elske : <b>Kjcerlighedcn</b> skjenkcs mere mild , hm og tvingende af <b>Qvindcn</b> end af Manden . Den gjhr hende mere taalmodig og vis og derfor mere indflydelsesrig. Hendes Ord crc mere mildnende , hendes Gloede mere vindende , hendes Pande mere tvingende , netop fordi den er mindre truende og frastodcndc . Den spcede Plante , bnn har at vogte , aabner lettere sin Blomsterkalk for Lyset i hendes Vlikke . Derfor er ogsaa en Moders Magt over sit speede Varn den sto ' rste moralske Magt paa Jorden . Og der ligger ei synderlig Overdrivelse i det Udtryk , at „ hun , som gynger Vuggeu , stvrer Verden . " <b>Sandheden</b> deraf vil blive tydelig af hvad som folger . Moderen har at vaagc over Charaktercn , medens den endnu af de forste Indtryk erholder sit Preeg og sin Form . De tidligste Yttringer bov Varnet af Tanke , Fplclse , Villie og Samoittigbed freintra ' de under hendes <b>Oine</b> . Hun har ikte blot at gjore med Vegemet i dets Barndom , men ogsaa med Sjeelen i deno forste Udvikling . Baade Sind og <b>Hjerte</b> ere i bendel Haand , nåar de gjore sine forste Anstrengelser til Godt eller Ondt . Af hende leere Vornenc at stamme de forste Ord , under bendes ledelse danne de sig de fyrste Begreber . De ere na ' sten bestandig i bendes Selskab, og uden at de <b>selv</b> vide det eller bnn meerker det , bestemmes af bende dereo gore eller flette Netning . Hun er deres forste Ereinpet , det forste Monster , boorefter de rette sig ' , Net og llret se de allerforst i bendes Handlemaadc . De ere bestandige Vidner til de Fortrin eller Feil , fom vise sig i hendes Ord , Gemvi og Handlinger . Uden at verre sig det bevidst , opdrager bun dem saaledes , ikke blot ved den Undervisning , hnn meddeler dem , men ogsaa ved Alt , bvad .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013100708190'target=_>Funcke, Otto, 1877, Apostelen Paulus til Søs og til Lands</a><h4>765</h4>i Stilhed , naar De virkelig har noget rigtigt at grcedc for , og trced saa atter med salvet Aafyn ind i Deres Families Kreds . Le og grced , syng og spil med de andre , saa vidt det vaa nogen Mande lader sig forene med <b>Sandheden</b> og Deres Samvittighed . Tilgiv , naar De bliver tromlet , <b>selv</b> naar man angriber Deres Tro , det , der er Dem allerhelligst i bed for de arme Spottere , der ikke veed , hvad de gjor . Tal ikke om Dem <b>selv</b> , gM ikte meget Vcesen af Dem <b>selv</b> , fornemmelig ikke af Deres smaa Lidelser , Deres Nerver , Deres Migrcene og deslige . Lad det derimod vcere Deres daglige Strceben og Virksomhed, at gjM godt overalt , at vise <b>Kjcerlighed</b> i smaat som i stort , at glatte de smaa Folder nd vaa Deres Medmenneskers Pander , at tjene og atter tjene dem , der treenger dertil . Bandlys al Forstemthed , Pirrelighed og andre Luner , lad al Knurren og alt Surmuleri fare , og vis i Gjerningen , hvad det vil sige : „ Voer taknemmelig altid og for alle Ting ! " Synes dette Dem for ringe ? O , tro De mig , det er ikke for ringe , det er for stort , for fvcert , ja , det er umuligt for Dem , undtagen naar De ret gaar ud og ind i den stille Helligdom , hvor Jesus bor . Og <b>selv</b> da vil De kun ufuldkomment kunne udftre denne Gjerning ! thi Kjpdets Sans i os os mange flemme Streger i Regningen . Men <b>selv</b> om De kun ufuldkomment udfprer Deres Gjerning fom en from Herrens <b>Tjenerinde</b> , faa vil De dog ved en ren Strceben efter det rette tjenende Sind efterhaanden vinde <b>Hjcerterne</b> . Og tror Folk forst , at Deres Christendom har gjort dem lykkeligere og elftvcerdigere, saa er meget vundet , og alt , det andet vil til sin Tid fo ' je sig af sig felv med en vidunderlig Hurtighed . Falder Deres Gjerning da end kun lidet i øjnene her pall Jorden , saa vil man kjende Dem saa meget bedre i Himlen , og hvad kan De forlange mere ? Jo lcengerc jeg betragter Verden , og jo mere jeg lcerer den at kjende , desto mere sinder jeg , at deres Tal er meget stort , som simpelt hen er kaldede til at gjore Livet en Smule behageligere, spdere og venligere for andre , omtrent paa

<h4>1053</h4>hvile i Gud under al vor Gjæren og Laden og at hente Fred for vor urolige Sjcel fra hans Stilhed , der er fuld af Liv , at lade vor Armod udfylde fra Evighedens rige Hav . Skriver vi da som de , der er i Gud og for Gud , faa vil vi ftrst og fremmest strive <b>sandt</b> . Mange Mennesker ere meget frommere i deres Breve , end de er i Virkeligheden ; iscer kommer der hen imod Slutningen en Syndflod af fromme Ord , som man let stjMner kun er Fraser . Men intet er modbydeligere end saadan en ordgydende Hellighed eller rettere Skinhellighed . Og den , der ftrst lyver i sit Forhold til Gud , han kommer ogsaa let til at lyve over for Menneskene . Jeg fplte mig overordentlig imponeret , da jeg engang kom ind til en <b>Dame</b> , der sad med glidende Ansigt og gjennemlceste et langt Brev , hun <b>selv</b> havde skrevet , og hun saa ftnderrev det lige for mine Ajne . Det undrede mig , da jeg vidste , at hendes Tid var meget kostbar . , Jeg spurgte , hvorfor hun tilintetgjorde fit eget Vcerk , og hun svarede : „ Der var lßet nogle Fraser og andre Usandheder ind med. " Det var en Troens Gjerning , jeg mener , at hun rev sit Brev i Stykker . Der bliver lFjet , smigret og hyklet grumme meget i Breve . Overdrevne Kj w r lighedsytring er hMr her til Skik og Brug . Man gaar langt videre i sin <b>Kjcerlighed</b> , end man vilde gaa , hvis man talte personlig med vedkommende . Staar man Ansigt til Ansigt med hinanden , saa vilde man ofte komme til at robe sig blot ved sit <b>Blik</b> og sit hele Vcesen ; men Paviret er taalmodigt og spørger os ikke : er det nu ogsaa <b>sandt</b> og oprigtig ment ? Men en Discipel af <b>Sandhedens</b> Konge fkulde skamme sig ved at lyve og hellere brcende et Brev end lcegge det i Postkassen med ' en urolig Samvittighed. Altsaai Den , der skriver for Gud , skriver <b>sandt</b> ; og videre : Den , der skriver for Gud , striver ogsaa naturlig , giver sig <b>selv</b> , som han virkelig er . For Gud og i Gud finder Mennesket sit inderste , egentlige Vcesen . Medens vi i Verdens Forvirring og Tummel saa ofte taber os felv , saa kommer vi for Guds Aasyn ret egentlig „ til os <b>selv</b> . " Der falder det unaturlige .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112301011'target=_>Hansen, Maurits, 1856, Mauritz Hansens Noveller og Fortællinger</a><h4>51</h4>gav han med Glede sit Minde til at Anton modtog et fordelagtigt Tilbud fra sin Universitetsven , Grev Rosenholm , at gjere Felge paa en vidensiabelig Rejse til Italien , Grcekenland og Orienten . Rosenholm tilbed ham fri Rejse ; men Falster , som ikke vilde vide sin Sen afhcrngig , udstyrede ham rigeligt , og saaledes forlod h , " , r ester et halvaarigt Ophold i Norge atter sin Familie , ledsaget af de varmeste Onsier og Velsignelser . At disse to modne Ynglinge rejste som Mend , viste Antons rige , sijenne Breve , og Falster rystede altid smilende paa Hovedet , naar hans gamle Fuldmegtig i de senere vansielige Handelsaar betenkelig raadede ham til at formindsie de store Remisser ! Madame Falster var en <b>Kone</b> , der i enhver Henseende fortjente at besidde en saa elskverdig Mand , og hendes stille , huslige Virksomhet , , den fortreffelige Opdragelse , hun havde givet sin eneste Datter og den faderog moderlese Mina , tildrog sig Alles Beundring . Rikke og Mina vare , hvad Aand og <b>Hjerte</b> angik , et <b>sandt</b> Sesterpar ; huldt opvorne i den skjenne Natur , tcet udenfor en af Norges mest kultiverede Steder , skjenne som Naturens fagreste Blomst , omfattede de hinanden med den inderligste <b>Kjerlighed</b> . Anton var ester sit Ophold hjemme , ferend han tiltraadte Rejsen , bleven et nyt Baand imellem Pigerne ; thi i de stille , fortrolige Timer , naar Rikke strommede over i Lovtaler over den nersten tilbedte Broder , dulgte ikke Mina for hende , at hun elskede den skjenne , talentfulde Yngling ; og Rikke betroede hende ethvert betydningsfull Ord af Antons Mund , naar han havde talt om Mina ; han siulde og maatte elske Mina ; derom var ingen Tvivl . Ethvert af hans Breve bekreftede ogsaa denne Mening ; thi i intet af dem var hun glemt , og <b>selv</b> det korteste og siyndsomste indeholdt et Par Linjer til hende . Begge de Gamle saa med Glcrde denne Tilbejelighed fremspire . Mina vil engang blive en <b>Kone</b> , som Du er , min Anne " , sagde Falster ofte , og trykkede sin Kones Haand .  Og Anton bliver en Mand og Fader som Du , Gamle " ; svarede den gode <b>Moder</b> og kyssede Falster emt paa Panden . > , Det skulde gjere mig ondt , om Anton lod sig fengsle af nogen fremmed Pige " , lagde hun saa til ;  thi tro mig , Minas <b>Hjerte</b> henger ved ham . "  Det har ingen Ned " , mente Faderen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016051048071'target=_>Bunyan, John, 1874, Pilegrimens Vandring</a><h4>768</h4>palle sammen for at forlade Byen , gik jeg og en Anden tilfældigvis hen for at beftge hende . Vi bankede som sceducmligt paa og traadte ind ; og da ui saa hvad hun foretog sig , spurgte vi hende , hvad det skulde betyde ; hun svarede , at der var sendt Bud efter hende , at hun ufortovet mantte begive sig kil sin Mand ; og derpaa fortalte hun os , at hun idrMme havde seet sin Mand at bo paa et herligt Sted blandt de Udpdelige , med en Krone paa Hovedet , spillende paa Harpe , spisende og drikkende ved sin Fyrstes Bord og syngende Lovsange for ham , fordi han havde fort ham did ofv . Nu , medens hun fortalte os om disse underfulde Ting , var det , som om mit <b>Hjerte</b> brcendte af Lcengsel , og jeg lovede HMoeligt ved mig <b>selv</b> : Er dette <b>sandt</b> , da vil jeg forlade Fader , <b>Moder</b> og Fødeland , for at Mge , om det bliver mig tilladt , med Kris tinde . Jeg spurgte hende altsaa videre , om hvorvidt det virkelig var <b>sandt</b> , og om hun havde Noget mod , at jeg fulgte med ; thi nu indsaa jeg , at der ikke lamgere kunde være Tale om at blive i vor Stad , naar man ikke vilde udscette sig for den visse Undergang . Dog tiltraadte jeg Reisen med tungt <b>Hjerte</b> , ikke fordi jeg var uvillig til at gaa , men fordi saa mange af mine Slcegtninge bleve tilbage. Og nu er jeg kommen med længselsfuldt <b>Hjerte</b> og vil , om det bliver mig forundt , ftlge med Kristinde til hendes Mand og Konge . Fortolkeren . Din Beslutning var god , thi derved har du vist din Tro paa <b>Sandheden</b> ; du er en Ruth , som af <b>Kjærlighed</b> til Noomi og til Herren hendes Gud forlod Fader , <b>Moder</b> og Fødeland for at drage hen til et Folk , som hun tilforn ikke kjendte . „ Herren betale din Gjerning ! og din LM stal være fuldkommen fra Herren , Israels Gud , til hvem du er kommen , for at være tryg under hans Vinger . " ( Ruth 2 , 12 ) . Maaltidet var nu tilende , og der blev truffet Anstalter for Natten ; <b>Kvinderne</b> sik hver sin Seng , og BMene laa sammen . Da Barmhjertig var kommen tilsengs , fik hun ikke sove af lutter Glæde , thi nu frygtede hun ikke længere som tilforn for at hun tilfidst vilde blive førskudt . Saa laa hun da og prisede og velsignede Gud , der havde været hende saa naadig . Om Morgenen stode de op med Solen og gjorde sig fcerdige til Afreisen ; men Fortolkeren vilde , at de skulde Wve

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013010208011'target=_>Kieler, Laura, 1869, Brands Døtre</a><h4>671</h4>sammen , „ Lys ! — Barn , hvem har lcert dig at tale saaledes , hos mig er intet Lys . " — „ Gud <b>selv</b> , dyrebare Fader , har ved sin Aand vist mig Lyset ; det er nu i mit — » Men hvem har sagt dig , at der fandtes et Lys ? " — „ Fsrst mit <b>Hjertes</b> Anelse og Lcengsel , elskede Fader , saaledes som du foler og altid har Mt det ; derpaa sendte Gud en Mand kort Jul , der viste mig Veien hen til det . Denne Mand , Fader " , Marias Stemme dirrede her , „ var Torkild Torkildson , og ham skylder jeg ncest den himmelske Fader <b>selv</b> en evig Taknemmelighed . Jeg har ikke sagt det for , Fader , sordi jeg vidste , at du var fortorvet paa ham ; men Fader , dyrebare Fader , tro mig , det vilde vcere saa godt , dersom du kunde tilgive ham . Hans hoieste jordiske Aiske er din Tilgivelse ; thi han ved og foler , bedre end nogen anden , at han feilede mod dig . Tro mig , Fader , han er en <b>sand</b> Kristen , o , dersom du kunde tilgive ham ! " Maria hcevede de foldede Hcrnder op mod Faderen , der stod mort og taus foran hende . Hendes <b>Blik</b> bad om Tilgivelse , — Tilgivelse for ham og Tilgivelse for sin Bon ; Maria havde aldrig for talt faalcdes til sin Fader , den Fader , hun kun havde betragtet med en übeskrivelig Mrefrygt . Hun bcrvede for det Svar , der vilde tomme . Svaret var imidlertid kun Taushed , urokkelig Taushed , hans Ansigt var bortvendt . „ Hvad er det for et Lys , der er i dit <b>Hjerte</b> , mit Barn ? " Dette havde Maria ikke ventet . Hun saa op med glad Mod , det straalende i ? ie spgte nt mode hans dunkle <b>Blik</b> . „ Troen paa den Værneret , Kristus ved sit Blod har forhvervet mig , Haabet om min Delagtighed i Hans livsalige Forjættelser , Haabet for dette Liv og for det tilkommende , <b>Kjcerligheden</b> til Ham , der elskede mig torst , og i Ham <b>Kjcrrlighed</b> til alle ; thi vi ere alle Syndere , og vi ere forloste ved hans Dod . " — Brand vendte sig pludselig

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112100014'target=_>Baur, Wilhelm, 1880, Billeder fra evangeliske Præsters Husliv i Tydskland</a><h4>570</h4>Andreæ ( 1586 — 1654 ) , — Speners Forløber ved det Alvor , hvormed han tragtede elter at forvandle Hovedets Orthodoksi til Hjertets levende Tro , Franckes Forløber ved Omsorgen for Børn og Gamle , Fattige og Syge , for Skolernes Opkomst og Kirkens Tjeneste , Zinzendorfs Forløber ved den aandelige Bevægelighed , hvormed han kastede sit fengslende Ord i bunden og übunden Stil ind i Tiden , aldrig fangen i nogen Maner , altid sig <b>selv</b> lig . Omendskjønt Theolog har han med en <b>sand</b> Umættelighed gjennemforsket historiske , geografiske , 6 iFteriß ^ e Værker . Med den mest anstrengte Aandsvirksomhed forener den livlige Yngling Gitarspil og Gymnastik , og Omgangen med 3 nedkere , Ilkrrnagere og ( ^ uld3inede beliver og styrker hans Sands for Hverdagslivets Virkelighed. <b>Selv</b> tilhørende den lutherske Kirke og Sønnesøn af en berømt Forkjæmper for luthersk Rettroenhed , staar han dog i Brevveksling med Amos Comenius , de bøhmiske og måhriske Brødres Biskop , beundrer Menighedslivet i det reformerte Genf , og naar han tegner os Kristenhedens Billede , saa som det foresvæver hans egen Sjel , gjør det det Bainme Indtrvlc paa 03 , som om de Troendes <b>Kjærlighed</b> fra de første Aarhundreder traadte os imøde . Midt ien Tid fuld af theologisk Sneverhjertethed en Mand med det videste Overdlik , midt i Meningernes Strid et varmt <b>Hjerte</b> , som Intet kjærere kjendte , end at bringe Hjerterne til at elske Kristum , midt i Krigens Nød en reddende <b>Kjærlighedens</b> Helt , træder hans fine Aand , let-bevægelig i Skrift og Tale , i Skranken mod Tidens Smagløshed . J . V . Andreæ var Sønnesøn af hin Jakob Andreæ , ved hvis Bestræbelse Konkordieformelen kom i3tand , ogße > n af Johannes Andreæ, Sognepræst til Herrenberg , senere i Konigsberg . Hans <b>Moder</b> , Maria Moser , al den berømte Slægt , som

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016111628008'target=_>, 1848, Evangelisk-christelig Psalmebog</a><h4>3299</h4>og takke din <b>Kjærlighed</b> for alle dine Velgjerninger, ja ogsaa derfor , at vi under din Vestjermelse have hvilet tryggeligen . I dit Navn begynde vi Dagen , og med Tillid vaa din Naade over os i Christo , ville vi ikke frygte . Vel vide vi , at vi leve ien ond Verden og <b>selv</b> ere svage ; vort eget <b>Hjerte</b> er bedrageligt , og Forfolerne ere mange . Ak , Herre ! forlad cs ikke , og overgiv os aldrig til os <b>selv</b> ; thi saa falde vi , og ' blive til Nop enten for Syndens Bedragene , eller Syndernes Svigefuldhed og Ondstab . O du , vor Frelser og Talsmand ! bed du for os , som hisset for dine Disciple , at vi , som ere i Verden , maae bevares for Verden og de forkmnkelige Lyster i Verden . Og du vil , thi du elster os , og har frikjobt os fra det Onde i Verden . Men ak ! at vi <b>selv</b> ville , <b>selv</b> vilde vange og bede , at vi ikke siulde falde i Fristelse . O du , som virker i os bande at ville og udrette alt efter din Velbehagelighed, du opvække vore Hjerter til at omgaaes varligen i Verden , og at holde os übesmittede fra dens fordærvelige Lyfter . Aabne du vole <b>Oine</b> , « t vi maae see alle de Strikker , man sætter for os , og gyse for den Afgrund , man vil styrte os i . Lad din den gode Aand bevare os fra Verdens Aand , at vi maae staae übe « vtrgelige , og engang faae Arv med dine Hellige . O Herre ! giv os Viisdom og Frimodighed til at fijelne imellem Onde og Gode , Lyst og Mod til at folge <b>Sandhed</b> og elske den , Had og Sinds Styrke mod Hyklerie og alt , hvad du hader . Dine <b>Oine</b> see og prove Menneskens Born . Ak , Herre ! aabenbar os hver Tanke og Lyst i os , som er dig mishagelig , og leed os vaa den evige Vei . Vcrr du denne Y 3

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016101948007'target=_>Merle d'Aubigné, J.H., 1877, Luthers Ungdom</a><h4>2042</h4>Medens Mary fordybede sig i Bogens Indhold , legede Sophy med Kattepns . „ O , nu kan jeg forståa, hvorfor Anna sendte mig denne Bog " , sagde Mary ved sig <b>selv</b> , medens hun standsede for at tsrre sine taarevcrdte <b>Oine</b> . „ Hun fortalte mig jo sidst , at Gud endnu gjør Kjcrrlighedsundere ; og er ikke dette Skrift en Optegnelse , af det mest opmuntrende . Her er en <b>sand</b> Historie om en Mand , der ligesom hine besatte med en ond Aand , var besat med Kærlighed til Drit . Han hendte sin Elendighed , han kjæmpede mod den ; han solte , at han med Sjcrl og Legeme gik Fordærvelsen unode ; og desuagtet , saa maatte han drikke , uagtet Drik var hans Ruin . Men Gud havde Medlidenhed med ham ; han havde hort hans Hustrues Bsnner og revet ham som en Brand ud af Ilden . Denne stakkels Synder var frelst forat blive til Velsignelse for tusinder , for at cere Gud ved sit Liv og sin Gjerning ; denne stakkels Synder er nu , jeg tviler ikke paa det , blaudt de hellige i Himmelen. Naar Vine Hall blev saaledes reddet , kan jeg da tvile paa , at min Mand ogsaa kunde blive det ? Var hnn ikke bragt til Gudsfrygt — levede han ikke i <b>Kjærlighed</b> til Ond , indtil Fristelserne ledede ham paa Afveie ? Hvem har saa mildt og kærlighedsfuldt <b>Hjerte</b> som min Giles ? Og har ikke ogsaa han en <b>Hustru</b> , der gnrder og beder for ham Nat og Dag ?

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009091503030'target=_>Pressensé, E. de, 1866, De tre første Aarhundreders Kirkehistorie</a><h4>381</h4>msrle og redselsfulle Side , personificeret i Erinnyerne , holder Sophocles sig til dens lyse og guddommelige Side . Man kan ikke tenke sig noget mere henrivende Billede puu Ande og Ustyld end det , hun hur frembragt i Antigane , der , forn hun <b>selv</b> siger , er fsdt for ut elske , ikke for ut hate , der , efterat hun hur fulgt sin gamle Fader i Landflygtighed og deelt hans Fattigdom , afsluner nt gjengjelde den hende forfslgende Fadermorder Ondt med Ondt og hellere vil dse end svigte sit <b>Hjertes</b> Indstydelsen Antigone har Mod som en Helgeninde og en ung grest Piges poetiske Svaghed , der , idet hun opoffrer sig , benreder den stjsnne Sol og det Brudeslsr , der skulde smykke hendes Pante ; men Hengivenheten og Fromheten lade hende ' vinde Seier over sig <b>selv</b> . Et guddommeligt Anndepust ganer igjennem dette ophsiede Verk , hvor <b>quindelig</b> <b>Kjerlighed</b> forener fig med Pligtens Heltemod ; mnn kunde kalde det et Streiflys „ der fru Grekenlands Himmel falder over de dybe <b>Sandheder</b> , den Gud , hvis Opoffrelse hur tert Menneskeheden <b>Kjerligheden</b> nt kjende , senere hnr nabenbaret for den . Ingen hnr med mere ophsiet Veltalenhed end Sophocles talt om den sedelige Forpligtelse, den udstelige , übsielige Lov , „ i hvilken der lever en Gud , som aldrig eldes . " Den religisfe Aand , der besjeler ham , bryter med en uforlignelig Skjsnhed frem i 52 dipus ' s sidste Ord , du den gamle landflygtige Konge i Dsdens Morke seer en hemmelighcdsfttld Klnrhcd stinne frem , der oplyser hnns ndstuttede Mne og bringer ham Lsfte om en falig Udstelighed IM . i Col . 1610 — 1625. )

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009080603010'target=_>Hagenbach, Karl Rudolf, 1860, nyeste Tids Kirkehistorie</a><h4>445</h4>rigtignok i poetisk Forklarelse , til det tydste Folks <b>Hjerte</b> . Vi gjore ham ikke nogen Bebreidelse derfor . Tvcrlimod : ligeoverfor de materialististe frivole Tcndentser , der tildeels vare blevne fremmede ved den Wielandste Skole , trengte Verden til en Vekkelse til det holere Liv , til en sedelig Styrkelse , en Henviisning til det Usynlige , der ikke kan gribcs med Henderne . <b>selv</b> om dette end nermest blot var et poetist-philosophist Ideal . Ligeoverfor hiin cgenkjerlige Anskuelse , der nedverdiger Dyden blot til en <b>Tjenerinde</b> for Nydclsessygen , trengte Verden til en vekkende Nost , der fremhevede Dydens Verdi som unfhengig af al udvortes Nesultat og plantede Begeistring for den i Sjelene , der overhovedet vendte <b>Blikket</b> bort fra Jordelivets Stov op mod Himmelen . Og det har Schiller gjort . Man vilde vere forblindet , ja utaknemlig , eller idetmindste uforstandig , hvis man ikke vilde anerkjende dette , ja i en vis Henseende anerkjende det med Glede , og det maa kun ansees for et godt Tegn . nnnr Ment » nf afgjort christelig Tenkemaade , som Albert Knapp i sit smutte Digt i Christoterpe for 1843 , have Mod til at udtale for Troende og Vantroe , hvad den tydste Nation har Schiller at takke for . Netop derfor kunne vi ogsaa dele Talerens festlige Stemning , da han ved Schiller- Statuens Asdekning i Stuttgard begyndte med de Ord : „ Gjennemtrengte af Beundring , af Wrbsdighed , men ogsaa af inderlig Glede , staae vi her forsamlede i Tusindviis foran det afslorede Billede af den store Digler , den dybsindige Folkelerer , Arbeideren paa Evighedens Tempel , den dyrebare Lcmdsmand , som er vor Etollhed og vor <b>Kjerlighed</b> " . Vi kunne , nåar vi tydeliggjore os Schillers Billede , med det samme Velbchng see op til den Pnnde .  der har grundet over Menneskehedens Bestemmelse og Tilskikkelse " ; med den samme Hoiagtclse see paa det dybe og klare <b>Die</b> . som sogte Skjonhcdcn i en indholdsrig Form . paa de veltalende Lebcr , fra hvilke en henrivende Sang udstrommede i fulde Toner , uden at vi derfor holde en Afguds tjene ste . en Bebreidelse , som vi ofte have hort udtalt . Men ungtet al denne Ancrkjendclse , uagtet al denne Beundring , tor vi . dog ikke

<h4>1320</h4>svermede Stolberg for politiske Friheds- Idealer . Paa en Reise til Schweitz i Aaret 1775 gjorde han Lavaters Bekjendtstab og blev med hele sin Sjel indtaget i ham . Om det Indtryk , som Lavater gjorde paa ham som Menneske , Familiefader. Omgangsven og Prest , strev han til Clandius i de meest begeistrede Udtryk ^ ) :  Naar min Phantafie er trettet ( nf Neisens Nytelser ) og vil udhvlle sig , snn forer jeg den ind i Lnvnters Hjem ; det gjor mig snn godt , ligesom jeg hver Gang har befundet mig saa inderlig vel , nåar jeg betraadte dette kjere Huses Dorterflel . Inderlig Fred , stille salig Nytelse opfyldte ofte min Sjel , endnu forend jeg fane ham , nåar hans kjere Bom glade kom mlg imode med deres Kjertegn, eller nåar jeg gjennem den halv aabne Dor til Sideverelfet sik <b>Oie</b> paa hans trofaste , milde , elskverdige <b>Hustru</b> . Og nåar jeg saae ham <b>selv</b> ! nåar han med hjertelig <b>Kjerlighed</b> sluttede os Alle i sine Arme ! O min kjereste Claudius ! De maa <b>selv</b> see ham ; de heteste Taarer vede Papiret , og jeg mangler Ord , nåar jeg vil tale om Lavater . og dog vit jeg fremstamme Noget om hnm , fordi ogsaa De holder saa inderlig af ham . Hvor vinder itte dette <b>Hjerte</b> ved nermere Bekjendtstab , dette <b>Hjerte</b> , der er ligesaa omfattende forn hans Genie , dette <b>Hjerte</b> , der brender af Nidkjerhed og dog er saa opfyldt af mlld Tolerancc ; der kan aabne sig for snnmnngc Venner , og for Enhver snnledcs , som om han var den eneste Ven . Bestandig er han eenfoldig som et Barn og dog hvor stor . hvor overstrommende , hvor dristig , hvor fri , hvor heltemodig , hvor reen i sine rene üblandede

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008090903039'target=_>Lamartine, Alphonse de, 1847, Girondisternes Historie</a><h4>940</h4>Ildstcenkene paa LEblet , syntes at udstynge ligesaa megcn Vmhed som Stolthcd igjennem de lange Vienhaar . Man fulgte hendes <b>Blik</b> , der ofte tabte sig i Nummet , som om man i dette med hende havde sundet den Indstydelsc , hun der forfulgte . Dette <b>Blik</b> , aabent og dybt som hendes Sjel , havde ligcsaa megen Munterhed som Ild . Man fslte at hendes Genies Lys kun var en Tilbagestraalen af en Dmhedsild, der siammede i hendes <b>Hjerte</b> . Ogsaa var der en hemmelig <b>Kjcerlighed</b> i enhver Beundring , hun vakte , og hun <b>selv</b> satte kun i Beundringen Priis paa Kjcrrliqhedcn . For hende var Kscerlighedcn kun straalende Beundring . Begivenhederne modne hurtigt . Ideerne og Tingene troengte sig sammen i hendes Liv ; hun havde aldrig havt nogen Barndom . I sit to og tyvende Aar bcsad hun Tantens Modenhed i Forening med Ungdommens Inde og Frodighcd. Hun strev som Rousseau , hun talte som Mirabeau . Dygtig til dristig Opfattelse og gjrnnemtcenkte Ndkast kunde hun paa engang i sit Bryst bevare en stor Tanke og en stor Fslelse . Lig Qvindcrnc i Rom , der rystede Revubliken ved deres <b>Hjertes</b> Bevcegelse , eller som gav og tilbagetog Herredommet med deres <b>Kjcerlighed</b> , vilde hun , at hcndcs Lidenstab siulde smelte sanunen med hendcs Politik , og at hendes Genies Ovsving siulde tjene til at hceve den , som hun elsiede . Hendes Kjsn forbod hende hiin umiddelbare Virksomhed , som kun den offentlige Plads , Talerstolen eller Armeen forunde Ma ? ndene i Offentlighedsregieringerne . Hun maatte forblive usynlig i Begivcnhederne , som hun lcdede . Vcere en stor Mands tilstorede Skjoebne , handle med hans Haand , hceve sig i hans Skjcebne , glimre under hans Navn , det var den eneste hende tilladte A3rgjerrighed ; men det er en om og ovoffrcnde 3 Ergjcrrighed , som forforer <b>Qvinden</b> , ligesom den tilfredsstiller det uegennyttige Genie . Hun kunde itle vcere andet af en politisk Mand end hans Samvittighet » og Indstydelsc ; hun ssgte denne Mand og hendes Indbildningskraft lod hende troe , at hun havde sundet ham .

<h4>1994</h4>lode ham i Rolands Huus mcerke deres Mening om denne Sag . Noland <b>selv</b> tiltog sig , ststtct paa sin Alder og sine Grundscetningers Strenghed , ' Myndighed til at erindre Dumouriez om hvor mcgen Agtelse en Mand i det Offentliges Tjeneste skyldte Ssmmeligheden og hvilket Erempel han burde vcere for de revolutionaire Sceder . Krigens Mand siog denne Bebreidelse hen i Spog ; han svarede Noland at han skyldte Nationen sin Nang , men at han ikke var forbunden til at opossre den hverken sine Tilboielighcder eller sin <b>Kjcerlighed</b> ; at han havde en Helts , men ikke en Puritaners Vegreb om Fcedrelandssindet . Ordenes Bitterhed esterlod Gift i Sjelene . De adstiltes med gjensidig Mistro . Fra denne Dag af udeblcv Dumouriez fra Madame Nolands Forsamlinger . Denne <b>Kone</b> , som kjendte det menneskelige <b>Hjerte</b> igjennem sit Genies og sit Kjons hoiere lustinct, bedrog sig ikte angaaende Generalens Sindelag .  Tv den er kommen da vi maae styrte Dumouriez , " sagde hun dristig til sine Venner .  leg veed nok / ' tilfsiede hun idet hun henvendte sig Noland ,  at Du ikke forstaaer Dig paa at stige ned til Intriguen eller Hcevnen , men hust paa , at Dumouriez maa i sit <b>Hjerte</b> pense paa at skade dem , som have fortornet barn . Naarman har vovet at gjsre en saadan Mand saadanne Bebreidelser , og har gjort dem ilden Nytte , da maa man ramme ham eller vente <b>selv</b> at blive rammet . " Hun felte rigtigt og sagde <b>sandt</b> . Dumonricz , hvis raste <b>Blik</b> havde skimtet at der bagved Girondisterne stod et Partie , som var endnu stcerkere og dristigere end deres eget , begyndte fra nu af at forene sig med lacobinernes Ledere . Han tcenkte med Grund , at Parliehadct vilde vcere stcerkere end Fcedrclandskjcerligheden , og at han , ved at smigre Nobespierres og Dantons Strceben efter Anseelse ligcoverfor Vrissot , P ^ tion og Noland , vilde i lacobinerne <b>selv</b> findc en Stette for Negjeringen . Han elskede Kongen , han beklagede Dronningen ; alle hans Fordomme vare for Monarkiet . Han vilde have vceret ligesaa stolt af at gjenoprettc Tronen , som af at ovretholde Nepublikcn . Han havde stor Dygtighcd i at bruge Menneskene , og alle Vcerkioier, der gjorde deres Nytte , vare ham gode . At ile forbi Girondistcrne , som , idet de beherskede Kongen , truede ham , og soge den Folkegunst , han havde nodig imod dem , lcengere

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112301014'target=_>Taine, Hippolyte, 1875, engelske Literaturs Historie</a><h4>1002</h4>Fortjeneste og min Skjønhed * ) . En gammel Hofmand gjer InHamora en X ^ serli ^ nsHserlllNriu ^ , og hun viser ham til Rette , pralende som en lykkelig Eventyrerske og plump som en Tjenestepige : » Belv om jeg ikke var Dronning , har De saa veiet min Ungdom , min blomstrende Skjønhed mod Deres Alderdom , som skjæmmes meer og meer * * ) . Ingen af disse <b>Heltinder</b> forstaaer at føre sig og træde op ; de forvexle Uforskammethed medVærdighed ogSandselighed med Ørnhed ; de hengive sig som Skjøger , de ere skinsyge som Friller , de ere smaalige som Borgerpiger , og de BiiMi6e ud som Fiskerkjærlinger . 2 va6 Heltene angaaer, saa ere de de üben3 ^ eligßte at alle Pralhalse . 1.60-nidas bliver først erkjendt som Arveprinds og derpaa pludselig forladt , og nu trøster han sig med denne beskedne Tanke : » Det er <b>sandt</b> , jeg er alene , men det var Gud ogsaa , før han Bll2bts Verden , og han fandt mere i sig <b>selv</b> end Naturen kunde give .... I mig seiv har jeg en Verden , stor nok til Skueplads for min Dyd * * * ) « Skal jeg tale om denne Konge for alle Storpralere , Almanzor, der , som Dryden <b>selv</b> siger , er skildret ekter Calpran66e'B-j-) Artaban ; han der hævner de forurettere , sprænger hele Hære og omstyrter Kongeriger ? Han der siger : » Maurerne have Himlen og mig til at staae den bi ... jeg vil fløite din tamme Lykke til mig som en Hund « ; han der bliver forelsket og taler om sin <b>Kjærlighed</b> i en Stil som denne : » Det er ham ! Jeg føler ham nu i hvert Led , som en ny Fyrste praier han med mit <b>Hjerte</b> , og kaster sit kongelige <b>Blik</b> over hver Krog i mit

<h4>1088</h4>hun er forvist fra Eders Seng og forjaget fra Eders Huuß , — og skjøndt kun er Cæsar ' s Søster , — saa er hun dog stadigt Eders . — Det er <b>sandt</b> , jeg har en BM , der foragter Eders Kulde , — som driver mig til ikke at sege , hvad I burde tilbyde , — men en Hustrues Dyd overvinder denne Stolthed , — Jeg kommer for at kræve Eder tilbage som min Eiendom , for først at vise Eder — min Troskab , og saa kræve , nei bønfalde , om Eders <b>Kjærlighed</b> . — Eders Haand , min Herre , den tilhører mig , og jeg forlanger den . « — Og da den ydmygede Antonius oprøres over den Tilgivelse , Octavia har skaffet ham , og siger , at hun uden Tvivl har tigget hans Liv at hendes Broder fattigt og lavt , saa svarer hun : » Fattigt og lavt havde jeg aldrig kunnet bede , — og tattisst og lavt havde min Broder ikke givet — Det er min haarde Skjæbne , — stadig at skulle lide ved Eders ukjærlige Misforstaaelse . — Men jeg bringer Eder slige Betingelser — at I ikke behøver at rødme over dem . — Jeg elsker Eders Hæder , fordi det er min egen . Aldrig skal det siges — at Octavia's Mand blev hendes Broders Slave . — Herre , I er fri , fri <b>selv</b> for hende , I er kjed af . — Vel vil min Broder kjøbe mig Eders <b>Kjærlighed</b> , — og kun give Eder Fred ved mig og gjennem mig , — men jeg har en Sjæl som Eders , jeg kan ikke modtage — Eders <b>Kjærlighed</b> som Almisse , eller tigge om det , jeg fortjener . — Jeg vil sige min Broder , at vi ere forligte ; — han vil trække sine Tropper tilbage , og I kan gaae bort — for at herske over Østen . Mig kan I lade blive i Athen , — ligegyldigt hvor ; jeg vil aldrig beklage mig — jeg vil nøies med det ufrugtbare <b>Hustru</b> Navn — og aldrig skal jeg falde Eder til Besvær . « — Dette er stort ; denne <b>Kvinde</b> har et stolt <b>Hjerte</b> og ogsaa en Hustrues <b>Hjerte</b> ; hun forstaaer at give og hun forstaaer at lide , og hvad bedre er , hun kan opoffre sig uden Pathos og i en rolig Tone ; det er ingen lav Sjæl , der har frembragt en Sjæl som denne . Og den

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006082900070'target=_>Brun, J.N., 1877, Gammelt nyt om &amp; af biskop Johan Nordahl Brun</a><h4>757</h4>Idealer ; men jeg tror mig ikke berettiget til at dadle alt det sanselige i dcn . <b>Kjerlighed</b> er af Gud , siger Johannes , og denne , som den eldste , maa derfor ogsaa vere det . <b>Selv</b> bygde Gud den fsrfte <b>Kvinde</b> af et finere Stof end Manden ; og kledte han Roser og Liljer med buntet Skjsnhed for at stjcenke Mennesket skyldfri Ojens Lyst , saa har han vel ikke uden Hensigt udbredt over det <b>kvindelige</b> Legeme saa meget tiltrekkende . Det gjor , det maa gjore , ja jeg tor sige , det var Skaberens Plan , at det skulde gjsre Indtryk . Naar nu dertil kommer Tenkemaadens , Sindelavets , Aandens Enighed , naar saa tvende , begge unge , begge elskverdige , begge ester frit Valg giver hverandre fsrfte Gang Haand og <b>Hjerte</b> , da indstrsmmer i dette Oieblik paa de forenede en Folelse , som med intet Billede i Naturen lader fig betegne , som synes at vere en Ndflydelse fra det store Vesen , vi kalder Gud , fra den Gud , Johannes kalder <b>Kjerlighed</b> . Om de da sad midt i Paradis , hvor skjon Natur havde opstillet tilskue alle sine Tryllerier , de saa dem ikke . Om liflige Toncrs sammcnstrsmmende Vellyst strsmmede ind paa deres Oren , de horte dem ikte . Om de stod midt under aecrocerauniske Klipper , de fandt dem ikke truende , de fslede kun sig <b>selv</b> , eller rettere : tabte sig ogsaa <b>selv</b> og alt i Kjerligheds übeskrivelige Fslelse . Da sansede vel ingen af dem at opsende nogen eneste Tanke til Gud , men siden , naar Luen holdt op at brage , naar den kun brendte klar og ren og stille , kunde de kaste et <b>Blik</b> tilbage og tenke : Saa god er Gud , han gav os dette Gode . Den Funke , som da faldt ned og antendte vore Hjerter , var nedstigende fra Lysenes Fader . Igjennem denne vor egen hellige Ild skimter vi ind til den Guddoins Veseu , om hvilken Johannes siger : Gud er <b>Kjerlighed</b> !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008092303003'target=_>Knigge, Adolph, 1857, Om Omgang med Mennesker</a><h4>862</h4>det noget , men dog ikke meget . Hun foragter maafke Dit Sslv , for desto sitkrere at vinde Dig <b>selv</b> med alt Dit Guld ; eller hendcs Temperament har mindre Hang til Penge end til Vellyst . Har Du bestandig sundet hende tro , trods mangchaande Fristelser , hvorved hun havde Lejlighed til hemmelig at bedrage Dig har hun viist kjoerlig Omhu , endog for Dit Rygte , for Din Wre , drager hun Dig ikke bort fra andre naturlige og cedle Forbindelser ; opossrer hun Dig Ungdom , Skjsnhed , Fordeel , Glands og Forfcengelighed , — nu vel , Anlceg og Temperamenter ere saa forskjellig ! blandede , og i saa Fald kan en Bolerske ogsaa have andre gode og elfkvcerdige Egenskaber ; men tro hende alligevcl ikke . En <b>Kvinde</b> , som ringeagter de ferste og helligste as alle kvindeligc Tyder , nemlig Kydsthed og Scedelighed , hvorledes kan hun noere nogen <b>sand</b> Wrefrygt for hsjere Pligter ? Jeg er imidlerlid langt ' fra at ville scette ' alle ulykkelige Faldne og Forserte i Klasse med hine foragtelige Skabninger. <b>Sand</b> <b>Kjcerlighcd</b> kan ogsaa fsre et vildfarende <b>Hjerte</b> tilbage til Dyden . Det er allerede ofte blevet sagt , at den som kjender Faren er mere sikker for at blive forsert end den , som aldrig er blcven fsrt i Fristelse. Men ved denne Slags Forseelser er en sikker og varig Forbedring bestandig noget tvivlsom , og Intet kan verre mere ydmygende og foruroligende , end naar man seer , at den man elsker bliver foragtct af Andre , eller naar man maa stamme fig ved en Forbindelse , som man hverken kan eller vil oplose . En <b>sand</b> og reen <b>Kjcerlighed</b> beskytter forsvrigt bedst for Udsvcrvelscr , og Omgangen med cedle , scedclige <b>Fruentimmer</b> stcerper Inglingens Sands for Dyd og Ustyld . saa at hans <b>Hjerte</b> , der er vcennet til noget bedre , foler sig vcebnct mod listige og frcekke Forlokkclser . — Forsvrigt er det uncrgtelig en stor Uretfardighed, at vi Mcend tillade os alle Sorter Udsvcevelscr, men ikke ville tilgive noget Fejltrin af Frucntimmerne, hvem vi dog fra Ungdommen af friste til at synde . Men hvad vilde der blive af det borgerlige Samfund og af hele Mennestesicegten , dersom denne

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014111908036'target=_>Brun, Johan Nordahl, 1841, Johan Nordahl Bruuns Hellige Taler</a><h4>226</h4>vrange Veie ; hvad scetter han ikke iVcerk , for at vinde ham tilbage , at forlade de Halvfemsindstyve , og gaae hen efter det Ene , at toende Lys og feie Huus , for at finde fin Penge, at anrette Gjestebud for en forloren Sen . Bestrider ikke alt dette den Guds alvorlige Villie : „ saa <b>sandt</b> som jeg lever , jeg vil ikke dens Dod , som synder , figer Herren . " Kunde du da , nåar du er i Nod , tamke paa den <b>Moder</b> , som ledte efter fin Son : nu soger min himmelske Fader mig , han raaber ester mig i fit Ord , ikke fordi han veed jo , hvor jeg er , paa hvad Vn jeg gaaer ; men fordi jeg veed det ikke <b>selv</b> . Han soger at fange mig , nu ved Velgjerninger , nu ved Kors , nu ved at lokke , nu ved at straffe , og en <b>Moder</b> kan aldrig vaere ommere for fine Born , og tale venligere til dem , end Gud til fine : „ Hvor ofte vilde jeg forsamle dig , ligesom en Hon ? forsamler fine Kyllinger under fine Vinger . " Laer derfor , o Menneske , Naaden i Naturen ! Lces i dit <b>Hjerte</b> , naar nogen Bmhed betager dig for en lidende Ven , og taenk : denne Sorg foler jeg , saa inderlig er jeg rort over dette ; men Gud elsker min Sjael endnu meget mere , „ thi Gud er <b>Kjcerlig</b> « hed . " Ja tykkes du Gud er fortornet paa dig , saa toenk : Han er fortornet , men som en <b>Moder</b> . Ogsaa i den Henseende vil vi see hendes Hjertelag : V ) Son ! figer hun , hvi gjorde du os dette ? Her er en Bebreidelse , men hvor moderlig! Maria havde havt Ret til at tale i en anden Tone ; mennaarhunharncevntdetkjcereOrd Son " , saa forråades hun strar af sit moderlige <b>Hjerte</b> . Son kunde hun nu kalde ham , og nisten som Paulus taler om Onefimo : min Son , hvilken hun atter foder med Smerte . " At forlade fine Forældre , at gaae , som det synes , paa fin egen Vei , at vcere rolig i alt dette , dette kunde vcere god Grund til Fortornelse.

<h4>1401</h4>Sandhed ny , mere indtrængende , bedre lysende ; men disse Grunde finde vi ikke altid , vi sege dem ofte omsonst , hverken hos os <b>selv</b> eller Andre ; ja ! vi kunne hverken hos os <b>selv</b> eller Andre opvcekke <b>Sandhedens</b> livagtige Folelser , nåar og hvor vi ville ; var det saa , da maatte vi som prcedike Evangelium , kunne omvende alle Vildfarende , troste alle Bedrsvede , og stabe Gl<ede hvorsomhelst vi forkynde det glade Budstab . Nu de modsatte Folelser , som vi kalde Fristelser , Anfegtninger, tunge morke Tanker ; raader du for dem , kan du hitte Rede i dem ? ere ikke Bevægelserne i dit <b>Hjerte</b> morke Taler ofte for dig <b>selv</b> ? var det ikke noget saadant Paulus folte , nåar han klagede over den Satans Engel som stog ham med knyttet N < eve ? vi begribe ganste vel , at Apostelen taler her Billedsprog , og at det var en Fristelse , en mork Tanke , som forfulgte , som plagede, som cengstede ham daglig . Vi opdage hos Men « nester , Mennester opdage hos sig <b>selv</b> mange deslige Tanker , Fristelser , som stride endog mod Naturen ; som den Tanke : at forkorte sine Dage , eller deres , til hvilke Naturen har indprentet os inderlig <b>Kjoerlighed</b> . Den kommer , man <b>moder</b> den , den folger , man afviser den , man hader den , og beholder den dog ; som Skyggen Le . gemet , saa er den med allevegne . Den Uerfarne kal » der det selvgjort Plage , Indbildningssyge , daarlig Drom ; men vi stulde ikke forhaste os i vore Domme , heller ynke vor Broder , heller bede til Gud for os <b>selv</b> : „ Led os ikke i Fristelse ! " eller , som vi synge : nåar vi og fristes ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2007101812001'target=_>Hillern, W. v., 1872, Ved egen Kraft</a><h4>1875</h4>en egen vemodig Ydmyghed over hendes hele Vesen , en stille Underkastelse , som kun <b>sand</b> <b>Kjerlighed</b> og <b>sand</b> Anger kan frembringe. Og hvor meget end Sorgen havde teret paa hendes Fylde og Friskhet » , saa erstattedes det rigeligt ved en lutret Bevidstheds Adel paa den blege Pante og ved det forklarede <b>Blik</b> i de omstyggede Dine , et <b>Blik</b> faa aandefuldt og glindsende , som om Sjelen fMe sig et andet Hjem og allerede saa det tilvinle sig i det Fjerne . Hun havde mere end gjerne indvilget i at forblive i Ziirich , thi hun stammede sig over at vende tilbage til Verden efter alt , hvad der var forefaldet , og forn Me var forbleien nogen Hemmelighed , og felv om dette Me havde veret Tilfeldet , faa var det Alfreds Ville at blive der , og dette var hende en Lov . Der var opstaaet et rMnde Forhold mellem denne <b>Moder</b> og denne SM , et Forhold , forn mellem Broder og SMer . Usigeligt havde hun lidt , saa ofte hun havde stuet Alfred ind i Diet , og han fMe det og lerte at Me Holmodighed . Med M Staanfel fMte den opvoxende Yndling at forjage Skamrødmen paa Moderens Pante ; men hun kunde aldrig undlade i ham at se Vidnet til sin Vanere og sin Dommer , og hun underordnede sig ham med en Udmyghed , en VillielMhed , som gjorde SMnen til Moderens übetingede Herre . Hvilket et vidt Gebet for Alfred til at udfolde sin Sjels hele Mtelmodighed ! Han gjorde det ogsaa . Noblesse oblike — og saaledes overbM begge hinanden i gjensidig <b>Kjerlighed</b> og Henfynstagen . Da Alfred var voren , turde han i <b>Sandhed</b> betragte sig forn Familiens Hoved, thi <b>selv</b> Tantene i det Fjerne vare afhengige af hans HM modighed , og han sMgete med Indsigt og Energi for dem uden at lade sig vildlede af deres Klager og BMner . Han var bleven Undling , da han endnu var Dreng , nu da han af Aar var yngling , var han en Mand .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014111408013'target=_>Wexels, W.A., 1846, Udsigt over Jesu Christi Liv paa Jorden</a><h4>1083</h4>Paa en af Jesu Vandringer traadte en ung Mand , en Vvcrstc ( maastee en Synagoge-Forstander ) , til ham og sagde : Gode Mester ! hvad Godt skal jeg gjore , at jeg kan arve det evige Liv ? Men Jesus vilde henvise den Sporgende , der i Frelseren kun saae en <b>Sandheds</b> ' Lcerer , til Kilden , hvorfra al <b>Sandhed</b> , Viisdom og Godhed udspringer , til den Gud , der alt i Loven havde givet Svaret paa hiint Eporgsmaal , og han sagde-derfor til ham : hvi kalder du mig god ? Ingen er god uden Een , nemlig Gud . Men vil du indgaae til Livet , da hold Budene ! Her , som da en Skriftklog ( see ovenfor § 6 ) fremkom med samme Sporgsmaal , som nu hiin Vverste , vilde Jesus lede den Sporgende til S elv-Erkj endelse . Oploert i Pharisoecrnes Skole til at gjore Forsijel mellem Budene , som mere eller mindre vigtige og nsdvendige , spurgte den unge Mand : hvilke ? Han vidste ikke , at Budene vel erc flere , forsaavidt de krceve forskjellige At tringer af det samme Sind , men at Lovens Krav i sit Vcrsen kun er eet : nemlig <b>Kjcerlighed</b> til Gud over Alt og til Nu-sten som sig <b>selv</b> . Men uden at indlade sig for Oieblikket i Berigtigelsen af den unge Mands sijceve Begreber i saa Henseende , svarede Jesus kun : du vced Budene ! ( det er de dig velbekjendte , almindelige Bud : ) distal ikke slaae ihjel ; du stal ikke bedrive Hoer ; du skal ikke stjcele ; du stal ikke sige falsk Vidnesbyrd ; crr din Fader og din <b>Moder</b> ; og : du stal elste din Nceste som dig <b>selv</b> . I sin Hjertens Oprigtighed , men stuffende sig <b>selv</b> under sine falske , as blinde Lcereres Undervisning indsugede Forestillinger om Betydningen af Budenes Opfyldelse , svarede den unge Mand : Mester ! dette har jeg holdt Altsammen fra min Ungdom af ; hvad fattes mig endnu ? Han fslte , at Noget fattedes ham under hans menneskelige Retfærdighed , men hvad det var , at det i Grunden var Alt , nemlig det <b>Hjerte</b> , der hcenger ved Gud alene og Intet begjcerer udenfor ham , vidste han ikke . Men Jesus saae paa ham med Alvor og Mildhed , elskede ham for hans usminkede Oprigtighed og indre Loengsel efter <b>Sandhed</b> til Salighed , og han sagde til ham : endnu fattes dig Eet : vil 95 «

<h4>1123</h4>og Mennestets og . den hele Naturs Legeme opstaacr og lcvcr i Christo Jesu evindelig , opstaacr og lever , for aldrig at hjemfalde igjen under Doden . „ De , som ere i Gravene , stulle hore Guds Sons Rost , og de siulle gaae frem , de , fom have gjort Godt ( de forhen aan deli g Opstandne ) , ti ! Livets Opstandelse . Det bor det Forkrænkelige at ifsres Usorkrcenkelighed , det Dodelige at ifores Udodelighed , og da opfyldes Ordet , som er strevet : Doden er opslugt til Seier ! " Og hvorledes er den opslugt ? hvorledes er Christus bleven Opstandelsen og Livet for os , der , vare dode i Synder og Overtrædelser? Ved <b>selv</b> at dse . Han dode , den Ene for Alle , for i sin Forsoningsdod at bortrydde Hindringerne for , at han kunde blive AlleS Liv ; han dode for Alle , paa det at Alle stulle leve i ham , leve for ham , leve med ham hans salige og hellige Liv . O , Disciplene vidste ikke , hvad de vilde , naar de vilde holde ham tilbage fra Doden , vidste ikke , at de derved vilde berove sig <b>selv</b> et evigt Liv ; de vidste ikke , hvad de vilde , naar de tcenkte paa at leve og < regjere i hans Rige , forend Syndens Sold af ham var betalt , forend han havde tjent dem til det Sidste og hengivet sit Liv til en Gjenlosningsbetaling for dem ; de vidste ikke , hvad de vilde , naar de tcenkte paa at smage hans Livs Samfund , forend de havde smagt hans 2 ods Samsund , at herliggjores med ham , forend de havde lidt med ham ! Forst Korset , saa Kronen ! Gjennem Doden til Liv.et ! saaledcs hed det om Mennesscns Son , saaledes hedder det om Enhver , som vil solge ham efter . Det er Tanker ved Lazarus ' s Grav . Hvor godt er det ikke at dvcele der med Doden i Livet og Livet i Doden for <b>Oie</b> ! Der vinder Troen Styrke , naar Hjertet er villigt , den Tro , der griber Christum og soger i ham Frelse fra det Onde , den Tro , der overvinder Verden og smiler i Doden . Ja , det villige <b>Hjerte</b> , o , maatte vi have det ! thi hvor Hjertet ikke vil troe , ak , der tjene jo alle Undere , alle Beviser paa ' Herrens Herlighed kun til at forhyrde det vantroe Sind , til at fjerne Mennesket mere og mere fra <b>Sandheden</b> og Livet . — Troen foder <b>Kjcerlighed</b> ; Hjertet hcenger fast ved Den , som er dets Salighed , og higer efter , i Ord og Gjerning at forkynde : jeg elsker dig ! Saaledes var det med Marias <b>Hjerte</b> , der i Jesus havde fundet sin Fred , sit Liv : hun salvede ham med den kostbare Salve , Billedet paa den <b>Kjcerlighed</b> , der udstrommede fra hendes Indre , og Herren annammede med Velbehag Hjertets Offer . Ogsaa vor <b>Kjcerlighed</b> vil han annamme , og han

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112201034'target=_>Taine, Hippolyte, 1874, engelske Literaturs Historie</a><h4>2341</h4>og inaa de elske lige til Døden . Men dette første <b>Blik</b> er en lidenskabelig Begeistring , og i dette <b>Kjærlighedens</b> pludselige Komme ligger der en uendelig lidenskabelig Fryd . Da Miranda seer Fernando , troer nun at 866 et overjordisk Vs6B6N , hun standser übevægelig , blændet al dette pludselige Syn , lyttende til de guddommelige Harmonier, som reise sig i hendes egen Sjæl . Hun græder , ved at 866 ham slæbe paa tunge Brændestykker ; med sine skrøbelige hvide Hænder vil hun gjøre hans Arbeide , for at han kan hvile sig imedens . Hendes Medlidenhed og hendes Ømhed rive hende med sig , hun er ikke mere Herre over sine Ord , hun siger , hvad hun ikke vil sige , hvad hendes Fader har forbudt hende at tale om , og hvad hun et øieblik i Forveien ikke vilde have tilstaaet sig <b>selv</b> . H6N663 Sjæl er fvlclt og fly der over , uden at vide det , lykkelig og undseelig over den Strøm af Glæde og nye Bevægelser , som en ukjendt Følelse har k ^ llit den med. „ Jeg er taabelig , at jeg græder over det , jeg er glad ved . " — . . Hvorfor græder Du ? " —  Over at jeg er uværdig til at tilbyde , hvad jeg ønsker at give , og endnu mindre fortjener at tåge det , som vilde bringe mig Døden , hvis jeg ikke fik det ... jeg er Eders <b>Hustru</b> , hvis I vil ægte mig ^ ellers vil jeg dø i Eders Tjeneste . " <b>Kjærligheden</b> griber dem endeligt og uimodstaaeligt , og omformer deres hele Karakteer ; den frygtsomme og kjærlige V6B < i6inona siger sig pludseligt løs fra sin Fader , i hans Nærværelse og foran det hele Senat ; ikke et Øieblik tænker hun paa at bede ham om Tilgivelse , eller trøste ham ; hun vil reise med Othello til Cypern , trods Uveiret og den fjendtlige Flaade ; det ene tilbedte Billede , som har faaet det fulde og absolute Herredømme over hendes <b>Hjerte</b> , fortrænger Alt andet fra hendes Tanke . Denne deres <b>Kjærlighed</b> fører da ogsaa til store Ulykker og dødelige Beslutninger ; Ophelia bliver gal , Julie dræber sig <b>selv</b> , og der er Ingen , som ikke seer , at dette Vanvid og denne Død ere nødvendige . Det er da ikke Dyden , vi finde i disse Sjæle ;

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015060948119'target=_>Brun, Johan Nordahl, 1843, Johan Nordahl Bruuns Hellige Taler</a><h4>1460</h4>os give Agt paa hverandre til Tilffyndelse til <b>Kjerlighed</b> og gode Gjerninger ! " Denne Vien-Salve anpriser Paulus , Ebr . 10 , 34. I disse opklarede Tider, da Mennesket bryster sig saa meget af sine Syner , da man uden Betingelse indwmmer en granskende For « stand saa megen Mre , var det dog nok et Sporgsmaal , enhver burde gjore sig <b>selv</b> : Hvad leder du efter ? hvad soger dit <b>Oie</b> midt i al den Mangfoldighed , som omringer dig ? Du , som ransager Skristerne ! Er det for at finde det evige Liv i dem , eller er det for at finde Plet « ter i Solen ? Du , som tcenker dig saa dybt ind i <b>Sandhedens</b> store Rige ! Er det og virkelig <b>Sandhed</b> , du soger? Eller leder du efter Vildfarelser i Andres Meninger , for kun at glimre <b>selv</b> paa den Enfoldigeres Bekostning ? Du , som samler paa Kundskab om Mennesket og dets <b>Hjerte</b> og Handlinger ! Hvortil agter du at bringe den ? Til at prale med Opdagelser , eller til at gavne , til at laste Skrobeligheder , eller til at rette dem , til at blotte Fcrdreues Bludsel , eller til at skjule Feil med Kjerligheds Kaabe ? Eller nåar du seer ud over det menneskelige Kjon , har du da forst brugt denne Apostelens Vien- Salve : Lader os give Agt paa hverandre til Tilskyn » del se til <b>Kjerlighed</b> og gode Gjerninger ! " Vi leve i et Selskab , den Ene kan ikke , bor ikke vcere den Anden li » gegyldig ; paa os <b>selv</b> bor vi give Agt fornemmelig , men ogsaa paa Andre , og da , hvor kommer det an paa hvad vi soge , hvad vi have sat til Maal for vor Undersogelse. Den , der reiser med bestemt Hensigt gjen « nem fremmede Lande , seer vel mange Ting , men giver kun Agt paa det , som er hans Hoved-Sag . Kjobmanden

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008092203001'target=_>Blædel, N.G., 1855, Prædikener</a><h4>538</h4>har <b>Hjerte</b> til at hjalpe , om end alle Hjerter og Done ere luktede for os her paa Jorden — saaledes at bygge paa Gud som paa en Klippe og tillidsfuld at staae paa denne Klippe , om end Våndene optaame sig og Stormene bruse hen derover , og saa i denne Tillid at taste al sin Sorg paa den almagtige Gud , for hvem Intet kan vare umuligt ' ) — gjor Du det ? hvor Mange iblandt os troe saaledes paa Himmelens og lordeus Staber ? Det er let vok , nåar Guds Fsrelser med os ere aabeubare og indlysende, saa <b>selv</b> det stsveste <b>Die</b> maa see Herrens Finger , at troe paa en Gud , der leder og styrer vor Gang ; men nåar hans Veie blive usporlige og hans Domme uraudsalige, nåar hans Tanker blive hsiere end alle vore Tanker , ja ! gaae endog lige imod vore Tanker , da at troe paa hans Viisdom , dette er det svare , og derved stal dog Troens Wgthed og Bestandighet ) proves . Naar vore Dnster opfyldes og Alt lykkes for os , da kunue vi maastee nok sige : Jeg troer paa en Fader i Himmelen , og vi mene maastee ogsaa , at vi virkelig have denne Tro ; men nåar han derover os det Kjareste paa Jorden , nåar han lagger det tunge , det langvarige , det smerterige Kors paa os , da at troe paa en kjarlig og barmhjertig Fader , der elsker os med en <b>Kjarlighed</b> , mod hvilken al menneskelig <b>Kjarlighed</b> er som siet Intet at regne , der barer os i sine Arme , som en sm <b>Moder</b> barer sit Barn — gjor Du det ? Hvor Mange iblandt os troe , haabe og elske med et Barns hele enfoldige Tillid ? Fordi saa Mange iblandt os ikke gjore det , derfor er det , at saa Mange iblandt os heller

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016101748017'target=_>Wildenhahn, August, 1860, Martin Luther</a><h4>733</h4>hvilken denne gav til Asrkebiffoppen af Mainz , og denne igjen til sin Sckrctcrr , der nu paa den modtagne Befaling hoit og offentligt oplæste Skrivelsens Indhold. Det var den af Pave Clemens den Syvende til Kardinalen ndfcrrdigede General-Fuldmagt , stilet til Keiscrcn og de førsamlede Nigsstcrnder . Efterat denne Form var iagttaget , tog Kardinalen <b>selv</b> Ordet og begyndte i et zirligt , latinsk Sprog en Tale , som pacmy overbeviste de evangeliske Fyrster om , at der af en saadan Forsamling ci kunde haabcs nogen fredelig Losning af deres hellige.og rctfcrrdige Sag . Han begyndte med store , truende Klager over , at Religionen i det tydffc Land , ved ondskabsfulde Gemytter , var bleven sat i en saa elendig Tilstand , som den cndnn nersten aldrig havde værct . Han talte om de mangeslags Kjcrttcricr , som vare opdukkede i Tydskland og som sogtc at forurolige og crrgre endog Petri lille Skib i Rom . Som Hovedaarsag til alle disse Grueligheder angav han Mangel paa <b>sand</b> christclig <b>Kjcrrlighed</b> og kaldte denne Tvedragt , som nu allerede tre Paver , L. eo den Tiende , Hadrian den Sjette og det nuværende Hoved for Christcnheden , Clemens den Syvende , paa ingen Maade havde kunnet hcrvc og bortrydde . Kilden til al den Ulykke , som i det forlobnc Aar var kommet over hele Tydskland . Derhos roste han ide stcrrkcste Udtryk den nuværende hellige Faders Omhyggelighed , og som i sit <b>Hjerte</b> folte den dybeste Bcdrovelse over disse Ting i Tydskland og opbod Alt , for igjen at skaffe Fred i Kirken . Dcrpaa henvendte han sig til de førsamlede Fyrster , og , med <b>Blikket</b> rettet paa de evangeliske Stcrnder , udtomte han sig i indstcrndige Formaninger om at ovhcrvc alle de opkomne Kjcrtterier og igjen oprette den gamle Enighed , hvortil jo , som han udtrykte sig , der var godt Haab , hvis blot Fyrsterne troligt vilde efterkomme alt Det , som Hans keiserlige Majestcrt med Hensyn til Religionen vilde befale dem . Derhos tilbod han saavel Pavens , som sine egne Tjenester og Mcrgling , og talte dcrpaa endnn meget , og uidtloftigt om Tyikckrigcn , dog mildere og mindre hadsk og fordommende end Pimpinelli . De stakkels , stakkels evangeliske Fyrster ! Saalc , dcs sad de da igjen domtc , fordomte , anklagede for alleslags Grneligheder og Skjcrndsclsgjerningcr , forhaanede i deres helligste Tro , for hvis Skyld de bode Gods og Frihed , ja Krone og Liv til Offer , domtc for Keiseren og Riget , som udstodte Spedalske af de Sundes Midte ! Deres sidste Haab var Sammenkaldelsen af et frit , almindeligt , christeligt Coneil , hvortil Keiser og Pave bestandig havde givet Lofter ; men derom sagde Kardinalen ikke et eneste Ord ; man kunde godt se , at Striden vel skulde afgjores , dog ikke i fri , christclig Overenskomst , og i gjensidig , christclig Rctfcrrdighed , men med Magt , ved Pavens og Kirkens Magtbud . Reiseren skulde befale , hvad der sknlde troes i Religionsfager ! Men det Smerteligste var endnu , at Kardinalen blot anklagede den ene Del , de Evangeliske , talte blot om udbrudte , fordommelige Kjcrttcrier , mcn om Vildfarelserne og Misbrugene i den papistiske Kirke , over hvilke endog de katholske Stcrnder havde klaget paa de tidligere Rigsdage , og hvilke endog den forrige Pave , Hadrian den Sjette offentlig havde indrommet og hvis nødvendige Afskaffelse han havde tilladt , talte han ikke et Ord .

<h4>817</h4>holde ved denne Bckjendelse , indtil det af den hellige , guddommelige Skrift bevises , at denne af os overleverede Bckjendelse ei er grundet i Guds Ord , Befaling og Orden ; men strider mod den hellige , guddommelige Skrift . Keiseren kunde ikke værge sig mod det Indtryk , som c » saa fast , afgjorende Vekjendelfe gjorde paa ham . Han saa , ffjondt mork og uvillig , dog med en næsten ufrivillig Wrcfrygt paa den Mand , der havde Mod til at lræde ham imøde . Men nodet af sin Samvittighed vedblev Kurfyrsten saaledes : " Eders Majestcrt kan , som en saa agtværdig Kriser , nu vist naadigst tcruke , hvornieget eu saadan Bcbreidelse maa smerte inigl " . " Onkel , " svarede Keiseren , " Alt vilde blive godt igjen , hvis I blot vilde tage eders Navn bort fra Bekjendelsenl " " Det kan ikke vare Eders Majestcrts alvorlige Villiel " svarede Kurfyrsten ' hurtigt . " Hvilken forfcrrdelig og fordommelig Skade for min Samvittighed , ja , hvilken stor Gudsbespottelse vilde jeg ikke lcesse paa mig , hvis jeg saaledes skulde forlade og fornægte en Lcrre , som jeg anser for overensstemmende med Guts Ord , Befaling og Orde » og for evig uforgjcrugelig <b>Sandhed</b> , ligesom om den ikke var netop Guds Ord , Orden og <b>Sandhed</b> ? Dette kan bvereen jeg eller mine Trocsfcrller gjore for Gud eller for Eders Majestcrt og de » hele Verden . Jeg tror heller ikte " vedblev han « åbenhjertigt , " at en saadan Anmodning kommer fra min unådigste Herres og Keisers eget <b>Hjerte</b> . Vi have store og mcrgtiqe Modstandere , som lcrnge og uafbrudt have anmodet Eders Majestcrt , indtil Eders Majestccts <b>Hjerte</b> er bleven bevcrget til at gjore mig eu saadan Begjcrnng . " Keiseren slog Vinene ned og hautede derpaa et flygtigt <b>Blik</b> paa sin kongelige Broder , der hidtil vel havde staaet i nogen Frastand som et stumt Vidne , men som dog ved sit Miucspil havde forraadt , at ha » tog den mest levende Andel i Samtalen . Ferdinand traadte nn nærmcre hen til Kurfyrsten og sagde : " Betænkcr Eders <b>Kjcrrlighed</b> da ogsaa , at vi for vor Del ogsaa holde paa Guds Ord og Orden ? Men er ikke Alt , hvad Eders <b>Kjcrrlighed</b> siger om sig , talt og rettet mod os ? Vekjende vi ikte ogsaa Christum ? " " Gud forbyde , at jeg skulde tvivle derpaa l " svarede Kurfyrsten . " Men min kongelige Herre maa vel bctænke , hvad der er Ret og Villigt i en saa alvorlig , hellig Sag . Et christeligt Sind formaar dog aldrig at tvinge sin Næste til at tro eller gjore noget Uchristeligt . Naar vi nu tro , at vor Lcrre , saaledes som vi offentlig have bet ' jendt den for Eders keiserlige Majestcrt og det hele Rige , er Guds Ord og Befaling , stulde da ikke vcre Modstandere <b>selv</b> domme saa meget og bekjende , at det aldeles ikke vilde anstaa ung og miue Trocsfcrller med god Samvittighed at frafalde dette ? Er det ikke en Pligt for vor Samvittighed , trofast at holde fast ved , hvad vi erkjende som Guds Ord , Guds Lcrre , Guds Befaliug og Ordcu ? Kuuue og tor vi handle anderledes , end at vi tillade og ikke hindre Saadant i vore Lande og Distrikter og at Alt afskaffes , hvad der ved lang Brug er opkommet og indfort alene ved menneskelige Anordninger , uden Guds Ord og Befaling ? " " Eders <b>Kjcrrlighed</b> taler om meuneskelige Anordninger ! " svarede Kongen .

<h4>962</h4>ja med Skræk og Forfcerdelse , da var Døden rigtignok en Ulykke . Mcn den Christnes <b>Die</b> ser ganste anderledes paa denne sidste Fiende , der tilintetgjøres i Døden . Bliktet paa Graven og Forraadnelsen er . for det christelige Sind et <b>Blik</b> i Guds vidunderlige Visdom og <b>Kjærlighed</b> , der endnu i Doven velsigner og gjor vel mod Begge , baade mod dem , som do , og dem , som blive tilbage , Christus har taget Magten fra Døden ! " „ O , dog er Døden ogsaa en Herrens Dom ! " svarede Leonardo med dyb Alvor . — « Jeg ved ikke , hvad det er for en Skræk , som netop gaar tgjenncm min Sjæl- Margrethe, ak , jeg er bleven saa bange , saa saare bange , for mig <b>selv</b> , for Dig , for min Fader ! Margrethe , sig mig , om Herrens Dom i min Faders Død var til Livet eller til den evige Død ! Forstoder ikke Gud ogsaa ved Døden ? Er det trostcligt , naar vi maa sige os <b>selv</b> , at med Legemets Død ogsaa Guds sidste Naades.Wieblik er forbi ? Og vi maa sige os det , fordi Gud ei kan drive noget Spil med sin Retfærdighed- Den hele Verdensstyrelse , al evig <b>Sandhed</b> , som gaar opvcektende , helligende igjenn ' em det hele Mennestefjsn , oplostr sig i et toMt Taagedillede , hvis Guds Retfærdighed ei er Sandhed. Gud holder , Gud inaa holde sin sidste Dom ved Døden . Ak , hvem siger mig , at min kjære , dyrebare Fader er domt til Livet og ikke til Døden ! " „ Din Tro paa Herren og paa hans Forjettelse ! " svarede hans <b>Kone</b> . „ Har Du ikke sagt mig . at din kjære Fader af ganste <b>Hjerte</b> har hengivet sig til Christus ? Hvilken Trost havde jeg med Hensyn til min Faders Død , hvis jeg ei vilde tro Christo ? Ja , min eneste Ven , lader os ikke klage og ssrge , som de , der ei have noget oaab ; men glæd Dig med mig over de Forlostes Salighed ! Hvor vederkvægende , hvor uudsigelig herlig er ikke Tanken paa den forventende Salighed , paa en Forløsning fra alle Onder , en Nydelse af den fuldkomneste Lykke ! Bliver itke ved et saadant Haab endog den haar « deste og ssrgeligste Tilvcerelse bersvet oens bittre Braao ? Maa ikke den sidste Smertenstaare borttsrres , naar Hjertet er saligt i Herren ? " „ Skjcend kun paa mig , min dyrebare <b>Kone</b> ! " svarede Leonardo . „ Skjcend paa min Vantro , min svaghed ! Ved dm Faders Dødsseng og Ligkiste var jeg en god Trøster , og havde fuldtop deraf ; men nu er jeg paa engang fattig og tom , og Vantroen vil gjøre mit Haab tilstamme . Du vil , at jeg stal være salig med Dig i Haabet ; men vt haabe kun , hvad vi ikke se , og hvorom Ingen fra sin Grav kan give den Levende Kund , stab . Hvis det nu ikke var <b>sandt</b> , hvad vi haabede ! Hvis den forventede Salighed vel var en stjon , men dog bedragerisk , forfængelig Drsm ? Hvis nu den Ene blot var den Andens Eftersnakker i denne Henseende , fordi Enhver har sit Velbehag deri ? " „ Stakkels Mand , hvorledes Du plager Dig ! " raabte Margrethe . „ Det forundrer mig , at Du ikke moisommeligt samler Alt , forat bevise for Dig <b>selv</b> . at vi ikke evigt do , men engang virkelig ville blive salige ! " „ Hvad vil Du sige dermed , Margrethe ! " spurgte Leonardo , „ Nu , " vedblev hun smilende . „ Vkal jeg bevise det for Dig , at vt engang ville leve evigt og i <b>Sandhed</b> og Virkelighed blive salige ? " „ Bevise ? " gjentog Leonardo forundret . „ Hvorledes vil Du gjøre det ? " „ Et . " svarede hun . næsten i en glad Tone , „ Du er bleven en Vantroende , og gjør , som Thomas , der blot vilde tro hvad han kunde se . Nu lceg Marte til , hvad jeg siger , Leonhard ! Jeg siger Dig : Sjælen vil og maa leve evigt ; thi den k ' an ikke do , fordi den ri har nonet Leycme , men er en Aand . Men Aanden dsr ikke . Nu , Leonhard . hvad siger Du til denne fortræffelige Trsst ? Er Du nu tilfreds ? Har Du nu Tro igjen ? Er ikke det en ganste scerdeles dyb Visdom ? " „ Du spotter mig ! " svarede Leonardo . „ Det kan være en meget lærd Tale ; men for den ssrgende og tvivlende Sjæl er den et Luftkastel . Bevis mig først , at jeg er en Aand . " „ Nu , " vedblev Margrethe venligt smilende : „ Jeg kan endnu paa en anden Maade bevise Dig , at vt ikke evigt ds. Ser Du , min Leonhard , Alt maa blive fuldkomment , ellers har det ikte noget Maal eller nogen Ende . Naar Du planter et Trce , saa er det ikke nok , at det frembringer Grene og Blade , men det maa vore og styrke sig saalænge . indtil det fremdriver Blomster og værer Frugter og er blevet et fuldkomment Trce .

<h4>970</h4>er netop opfundet af saadanne taabelige Mennesker , som jeg <b>selv</b> er ? Hvorledes vilde Du da helbrede min Vantro og bekjæmpe mine Tvivl ? " „ Nu du blinde Mand ! " raabte den Blinde med glad Rorelse og strakte sine Hænder ud for at finde og gribe sin Mands Haand . „ Foler Du da ikke , at jeg <b>selv</b> spotter over al min Visdom , med hvilten jeg har villet fore Dig til Troen ? Foler Du ikke tgjennem din hele Sjæl , hvorledes Gud gjor denne Verdens Visdom til Daarstab , og tlltntetgjør de Vises Visdom , naar de ville udgrunde og bevise det Himmelske og Evige ? Siger ikke dit hele <b>Hjerte</b> Dig , at Du er forladt , udkastet i den ode Drken , saasnart Du slipper Troen . Hvorledes vil Du med din egen Visdom igjenfinde det , som er gaaet tabt for Dig , naar Du ikke lamgere tror Guds Ord ? Intet og Ingen paa Jorden kan lære Dig at tro , uden Guds hellige Aand alene , der oplyser dit <b>Hjerte</b> , foreholder Dig din Livs . og Hjertensd , lader Dig stue ind i din Elendigheds hele Dybde og driver Dig op til Herren og Hjælperen ud af al Legems « og Sjælensd . Min dyrebare, kjære Ven , " vedblev Margarethe bevæget : „ Jeg ved , at Du ikke alvorligt tvivler , at kun dit barnlige <b>Hjertes</b> Smerte over din dyrebare Fader , som dit legemlige <b>Die</b> ikke mere skulde se , piner Dig med saadanne Tanker ; jeg ved sikkert , at Du meget snart , førsonet med din Gud og Herre , igjen vil finde Dig tilrette i den evige Kjærltghed og Naade , — men erkjend det med mig paany , — mangle vi Troen , da ere vi fortabte , og al Verdens Visdom kan ikke lægge et Korn Trost og Fred i vor arme Sjæl . , Thi Ordet og Korset er med engang en Daarlighed for dem , som skulle blive fortabte ; men for os , som tro , er det en Guds Kraft til Salighed . Vel os , min sode , kjære Ven , at vi have en anden Borgen for vor Salighed , end denne Verdens Visdom , nemlig vor Herre Jesus Christus , om hvilken vi vide , at han er kommen til Verden , for at gjore Synderne salige . Naar denne min og din Herre for mig og Dig beder : Fader , jeg vil , at hvor jeg er , stulle ogsaa de være , som Du har Hivet mig , paa det at de kunne se min Herlighed , som Du har givet migi saa staar jeg med de to Ord : jeg vil ! Alt ned , hvad der kunde reise sig derimod . Naar min Gud , min Herre og min Forlsser siger : jeg vil , saa er ingen Tvivl mere mulig ; thi hvad min Herre siger , det er <b>sandt</b> , og hvad min Herre vil , det ster , og mig tilkommer det , at jeg tror derpaa . Og medens de Andre pine og plage sig , og komme dog ikke ud af sin egen Visdom , sidder jeg rolig og trostig under den Hoiestes ' Beskyttelse . Og naar jeg tror min Herre , naar han siger : Gud rc- Aja > rligl ) eden , saa tror jeg ham ogsaa , naar han siger : Jeg vil at Du skal va > re hos mig ! " Da Margaretbe her taug , sank Leonhard med bævende Taarer i Diet til hendes Bryst og sagde . „ Jeg arme , arme Menneske , der vilde undlobe min Guds baand ! Ak Margarethe , jeg har bestandig troet , at Gud har givet mig Dig som en god Engel til at lede og fore mig paa min Livsbane ; men fra idag ved jeg , at Du for mig er en Guds Gesandt ! " „ Synd ikke med din <b>Kjærlighed</b> ! " bad hans <b>Kone</b> og græd af Glæde ved hans <b>Hjerte</b> . Og den gamle Mester Urban traadte ind og sagde : „ Born , jeg har hort Alt / hvad I have sagt . Og nu , da dit <b>Hjerte</b> igjen er trostet og glad , min Son , saa erfar da og lces det her i din salig Faders sidste Skrivelse , som han har ladet afgaa til mig tre Dage for sin Død : Du stal blive hos os , opsiaa dit Hjem i vort kjære , tydsse Fcedreland; mit Hus stal være dit Hus ! Forat Du , min Son , skulde glad indvilge i dette , er din kjære Fader dod glad og salig ! Hvad vil Du nu gjore ? " „ Guds Villie sie ! " svarede Leonhard og bad stille og med saligt smilende Ansigt .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016051048044'target=_>Halling, H., 1858, Søndagsbog for Søfarende</a><h4>1934</h4>selv os foder , Bespiser os med Himmelbrod Og til sin Grcesgang leder ! Vort <b>Hjerte</b> vedcrqvceger Han Og Icedster det med Livsens Vand Af Naadens rene Kilde . 2. For <b>Sandheds</b> Vei den Hellig Acmd Oplader klart vort <b>Oic</b> , Og er vor hulde Trostcrmand I al vor Sorg og Moie . Thi have vi saa frit et Mod , Om Døden end for <b>Oie</b> stod : Din Stav , o Gud , os leder ! - 3. Din Godhed og Din Miskundhed Os folger her i Live , Med Glæde og med Hjertefred I Herrens HuuS vi blive , Her i den christnc Menighed Og efter DodenS Usselhed I Himmerig med 3 Ere . 4. Dertil os hjcrlp , o Jesu Christ ! Lad vore Hjerter brcende af <b>Kjærlighedens</b> Ild forvist , Saa vi Din Fader fjende Med Dicsog med Din Hellig Acmd ! O hold os , Herre , ved Din Haand , Til Scieren er vunden ;

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112000014'target=_>, 1862, For Hjemmet</a><h4>1249</h4>Rigsgrevinden saae smilende , men med fugtige Dine ned paa hende . « , Er det virkelig saa . Du lille Trodsige ? " spurgte hun .  Bryder Du Dig viikctig endnu Noget om din gamle Bedste , <b>moder</b> ? Jeg meente , det var reent forbi mellem os , at Du var meget vred paa mig . og at din <b>Kjcerlighed</b> var — hos Andre " . —  O , Bedstkmoderl " —  See engang paa mig , Margarethe — men ikke med disse hsitidelige Dine , de gjsre mig bange ! Nei . med det gamle , trofaste , inderlige <b>Blik</b> ! Saa ! Siig mig nu — har Du dette — Menneske saa kjcer , ret af dit inderste <b>Hjerte</b> ? " —  la . Bedstemoder " . —  Har Du fuld Tillid til ham ? In . destaaer Du for ham , for hans Redelighed , hans Trofasthed , hans Kraft ? " —  la . Bedstemoder " . —  Og dog har han vaklet — og kjcrmpet — saa lange ! Dog har han <b>selv</b> sagt til Hugo , at han helst skulde snske , han var dsd . thi han lunde saa aldrig faae Dig . Er det kraftigt , retcligt . trofast mod Dig ? " — — det var for . Det maa have varet

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111903009'target=_>Hansen, Maurits, 1858, Mauritz Hansens Noveller og Fortællinger</a><h4>1639</h4>senere kom Ferdinand Wagtler til Byen , ester at han havde taget sin anden Eramcn . Han havde som Discipel i Kristiansands Skole nydt meget godt hos Reineck , som den Gang boede i Stiftsstaden , og var understøttet af Konsulen under fit Ophold i Kristiania , da hans fattige Forcrldre vare dsde , og han ingen Slcrgtning havde at ty hen til , Faberligt modtog Rcineck den unge , stinke Student , — sosterligt bebandlede Karoline ham . Hans Ophold forlcengede sig fra Uge tilNge ; og stedse blev han mere mlsfornsjet med den stolte Britte og hans paa Spekulation grundede Forbindelse med Froken Reineck , stedse tiltalte Pigens Elssvcrrdighed ham stcrrkere ; og med et ingenlunde undssyldeligt < . ! , ovu ! eiie antog han en Weithersi Nolle . Vel var Froken Reineck for lidt Romanheltinde til at bringes ud af sin Rolighed ved den Lidcnffab , der nu og da glimtede frem igjennem Wagtlers stjaalne <b>Blik</b> eller undertrykte Sukke : imidlertid var hun <b>selv</b> alt for uforstilt og cerlig til at tro , at den Letsiudige drev Spil med sine Fslelser og morede sig med at bilde hende ind , at han var ulykkelig . Saaledes beklagede hun ham af gansse <b>Hjerte</b> og felte sig , uden at vide det , smigret ved en saa « del , faa fordringolos , saa haablss <b>Kjcrrlighed</b> hos en ung Mand , der , — hvad Aands- og Legems-Fortrin angik — rigtignok dannede en paafaldende Modsa > tning til hendes Englcrnder . Karoline onsicde intet mere , end at faa Wagtler til at rejse ; og , da han ikke forstod eller vilde forståa de delikate Admonitioncr , den fint felende Jomfru vovede at tillade sig , besluttede hun fig til at betro sig til sin <b>Moder</b> . Den gode Matrone blev hsjlig bestyrtset ; da hun altid havde holdt af Ferdinand , tog hun Sagen fra den vmmeste Side og tcrnkte blot paa , hvorledes den stakkels Outs Hjcrtesaar skulde lcrges . Vel bevandret i Romaner og Skuespil fandt hun en vis Fornejelse i ved denne Lejlighed at give den vcrrdige <b>Moder</b> . Saa snart Karoline havde beroliget sin Samvittighed ved at betro den delikate Sag til Moderens Erfarenhed , overlot » hun sig til sit scrdvanlige lette meddelende Sind og troede at burde vise Ungdomsvennen dobbelt saa megen Velvilje , da der nu snart forestod ham en smertelig Afsiedstime og en maaste langvarig Hjcrtesorg .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013042608020'target=_>Funcke, Otto, 1879, Glæde, Lidelse, Arbeide i Evighedens Lys</a><h4>942</h4>sin Tid hostet fabelagtigt Bifald . Men dersom det forholder sig , som han siger , da er den barmhjertige Samaritan , som staber Glcede , Fred og Forsoning i det <b>Hjerte</b> , hvor Forbitrelse og Fortvivlelse har sit Hovedkvarter , storre end alle store Teologer og Filosofer, som mangler dette Samaritanersind . " Den , som har forvandlet et <b>Die</b> , fuldt af Smertestaarer , til et <b>Oie</b> fuldt af Glcedestaarer , han har frembragt mere end et gront Straa , Ja sandelig , dersom det er Guds hellige og salige Villie , at det ene Menneske stal fsre det andet til Frelse , saa er der i Verdens Elendighed en Mark , der er „ hvid til Hosten . " Dersom du'engang har ligget syg og forladt i Smerte og Angst , og da oplevet , at et Menneske , drevet af Medlidenhed og <b>Kjcerlighed</b> , satte sig ved dit Leie , tog din Haand mellem sine og viste dig ved hvert Ord , som han sagde , at han forstod dig , at han led med dig , at han tcentte paa , hvorledes han kunde vcere dig til Hjcelp og Trost , ikke <b>sandt</b> , et saadant Menneske var for dig fom en Engel fra Himmelen , <b>selv</b> om det havde et hcesligt Ansigt som Lea og en Skikkelse som Kroblingen Mefiboseth ; dit <b>Hjerte</b> blev atter blodt ug dit <b>Die</b> vaadt , og var du begyndt at tvivle paa din Gud , saa kunde du nu atter tro paa hans <b>Kjcerlighed</b> , En Socialist „ af det reneste Vand " ovtraadte engang i et Partimode meget bestemt til Forsvar for Presterne . „ Hvilket Under ! " horer jeg Lceseren raabe . Ja visselig ; i Socialdemokraternes Partimoder pleier man ikke levne Prestcrne megen Mre . Men hin Presteven sagde til sine Kammerater: „ Hor , da min <b>Kone</b> taa paa Dodsleiet , og de fleste af mine Born ogsaa var syge , da var det en Prest , som jeg i sin Tid havde forncermet , som bragte mig Hjcelp ! blandt andet forte han til mig , hjcelftelofe og fortvivlede Menneske ! en Diakonisse , en Engel i <b>Kjcerlighed</b> , forfiktrer jeg eder , og siden siger jeg om disse „ Hytlere " : „ Gid der var mange af dem . " Ligger der ikke en uhyre Auklage i disse rosende Ord : „ O , hvor megen Forbandelse mod Gud , hvor

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008110603024'target=_>Vig, Ole, 1859, Enkelte af Ole Vigs Arbeider til Folkelæsning</a><h4>65</h4>Senere tjente han som Icetergut . Vigs <b>Moder</b> Marit var derimod Gaardmandsdatter fra Gaarden Storvalstad . Hendes Bedstemoder igjen var kommen fra Helgeland , og siges at have virret af Finnecet ; hun skal som Tjenestepige have flyttet med en Prcest Raklev til Stordalen . Ole Vigs Forceldrc omtales som fattige , men saare retsindige og i en sjelden Grad kristeligsindede Folk , der under trange Kaar opbrog sine Born til Gudsfrygt og Arbeidsomhcd. De havde ialt 4 Born : 3 Sonner og en Datter ; as disse dode imidlertid de to Sonner allerede tidlig , saa det nu kun var Ole og Datteren , der var Gjenstand for Foreldrenes Omhu , af hvilke iscer Moderen udmcerkede sig ved sit fromme Sind , sin oplyste Forstand og sin kjcerlige Agtpaaaivenhed paa sine to Borns Udvikling og Opdragelse , ikke blot hvad Legemet , men meget mere , hvad deres <b>Hjerte</b> og Forstand angik . Derfor kunde den Afdsde , saa sjelden han end forovngt talte om sit Barndomsliv , dog aldrig noksom omtale sin <b>Moder</b> og det Meget , han havde hende at takke for . Og mangengang beklagede han , hvormeget der dog under Livskampen var gaaet tabt af den Barnetro, der var rodfcestet og opvokset under hendes kjcerlige Pleje . Hendes Dsd , som indtraf omtrent et Aar for hans , gik ham derfor inderligt til <b>Hjerte</b> . Det Brev , han strev til hende paa hendes Dodsleje , strommer over af en barnlig <b>Kjcerlighed</b> , der endnu havde bevaret sig lige varm , ligesom han deri ssger at dcempe Vemoden over det forestaarnde Tab ved det Oftstandelsens baab , der altid var hans Trost , naar Doden og Morkct syntes ham at ville kue Lyset og <b>Sandheden</b> . Ligesaa vakker som den er i sig <b>selv</b> , ligesaa skjon en Aabenbarelse af Barneaanden under"Kjcempestjoldet, er folgende lille Itring i det ncevnte Brev :  Gak nu hen forst , du <b>Mor</b> , saa vil jeg se at blive mere himmclsindet , og saa kommer jeg ester . " Ligesom Faderen og iscer Moderen var bekjendte der i Bygden sor meget begavede Folk , saa er der et Trcek , som ikke er saa ganske uinteressant at lcegge Mcrrke til , nemlig at hans Farfader , der er dod for lccngere Tid siden , endnu omtales af Alle deroppe for sit , , udmcerkede Nemme " og sin  store Lcesning . " Naar han saaledes kom hjem fra Kirken , kunde han gjentage hele Prcekenen , og og stal have besiddet scerdeles Gaver til at  lcegge ud . " Han synes i alle Dele at have havt meget tilfcelles med

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081303012'target=_>, 1863, Bibelen, eller Den hellige Skrift</a><h4>2956</h4>Vedhob , stal antcendes , forat ncrre Ilden under Gryden . Mm dette er den guddommelige Vredes Ild , den Doms Lue , som fuldbyrdedes 10 ved Chaldcrernc . — At Rjsdet fortcrres , og osv. Kjod , Urter og Marvbcn ere de forskjellige Steender blandt Jerusalems Indvaanerc , som nndcr den Nod , Beleiringen forvolder , ved Guds Dom stulle om » 11 komme i Staden som i sin Gryde , — Og ftrt den tom . Om end tom og ode er dog Jerusalem det Navn , som er bestemt til at holde Folket sammen og engang igjen forene det . Dommen over Stadens Huse , som opbrcendes , trceffcr virkelig Folkets <b>Hjerte</b> som hcrngcr ved 14 den . I den bliver Folkeaanden <b>selv</b> truffen og straffet . — Skulle de.  De , " nemlig de , som Herren har forordnet til at fuldbyrde Dommen ; oisse stulle domme dig og give dig Gjengjeld for den Maade , hvorpaa Du har levet , og for det , du har gjort . Herren <b>selv</b> fuldbyrdcr 16 sin Dom gjenncm dem . Sml . Ez . 7 , 27. — Dine Mnes Lyst . Det Kjcrrestc , dine <b>Oine</b> se . Saaledes betegnes her forclobigen Ezechiels <b>Hustru</b> fsml . V . 18. ) , fordi Herren tog hende fra ham , ikke fordi hun var hans Hustrn , men fordi hnn var hans Kjcereste . Hvad den elstede <b>Hustru</b> er for Manden . det er Helligdommen for Israels Folk ssml . V . 24 ) . Men Hustruen og Helligdommen kaldes Vinenes Lyst som detKjcrreste blandt de synlige Ting . Profeten har Herren langt kjcerere ; men han er usynlig . Ned en Plage . Plage er egentlig et grcrst Ord , som er ganet over i det latinske og derpaa i det tydste og vorr Sprog , og betyder Slag ; Gud maa da tcenkes til som den , der staar . Plage er et uf Gud med bestemt Hensigt tilstikket Onde , hvorved han vil foraarsage Smerte ; derfor scvdvanligvis et Straffeonde , > for ved det at fore Syndere til Smerte over sin Synd og Erkjendelse - as sin Skyld . Lzcchiels <b>Hustru</b> blev slagen , saa hun dodc , for ved dette Slag , som ncermest traf hans <b>Hustru</b> , at tilfoie ham Smerte , men ikke forat straffe ham , men forat han for Israels Folk stnlde vcere et Forbillede paa . hvorledes det skulde blive straffet ved den af Herren tilstikkcde Ddelcrggclse af dets Helligdom , Templet . Ezechicl maatte lide dette , ligesom andre Lidelser ved hans profetiske Embede , som Guds Tjener og Redstab for Israel , forat fore Folket til en gavnlig Bod . Saalidct sparer Gud sive Tjenere , og de bcere det gjerne , fordi de vide , at Herren i sin Tid vil gjore Alt over al Maade godt igjen , og fordi de bestandig i <b>Kjcerlighed</b> og Tillid ere beredte til at offre ham Alt , hvad i ? han fordrer . — V . 17 : Han forbydes ikke at sorge i sit Indre ; men kun hcmmeligen og i Londom stal han sorge , til Tegn paa , at Israel i Trceldom under fremmede Folk ikke vil vove lydcligen at yttre sin Sorg over Helligdommcns Ddelceggelse . Sml . B . 22. 23. Og jeg talede ( dette ) , nemlig Talen V . 3 — 14. Og jeg gjorde osv. Jeg afholdt mig i Folkets Paasyn fra enhver Dodsklage ved min Hustrus 19 Lig og fra ethvert Sorgetcgn , som er i Skik og Brng . — V . 19 : Folket , som paa Efterretningen om hans Hustrus Dod havde forsamlet sig hos ham , forat klage over den Dode , spurgte Ezechicl hvad Bctydning denne hans uscrdvanlige Adflrrd havde for det . Det spurgtc , fordi Profeten allerede flere Gange paa Herrens Befaling havde gjort 2 i sig til Typus paa Jerusalem for det . Sml . C . 4 og 12. — Som I lade ( efter Eder ) , som dcngang cndnn vare blcvne tilbage i le-23 rusalcm i god Behold . — Forsvinde i Eders Misgjerninger , for

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008102303004'target=_>Trollope, Frances, 1847, Constance Ridley</a><h4>1612</h4>det — om hans Fortrin trodsete alle Anstrengelser af tenne sildigfodtc Bestedenhed og Mistillit» til mig <b>selv</b> — om jeg var beredvillig til at indromme , at jeg idetmindste ikte tunte tåge feil af ham — at han i <b>Sandhed</b> eiede Alt . hvad edlest og bedst fintes i den menneskelige Natur — Alt , hvud der er meest indtagende , meest egnet til at indgyde <b>Kjerlighed</b> . Beundring og Agtelse ; om jeg indrsmmcte Alt tette og saae ham tna-lc for mine Fsdder og hsrtc ham tilbyde mig teite edle <b>Hjerte</b> , tenne ophsicte Aants Ledsagelse paa Livets Vandring , kunde jeg da vel modtage hans Tilbud ? O ! nei , nei , nei , — det var umuligt ! Hvis han sagde « Constance , jeg clsscr Dem — vil De vere min <b>Hustru</b> ? « — ssulde jeg ta svan ham : « Min Herre , jeg forlovede mig med Mr . Mortimer sidste Onsdag , forti jeg var bleven ganske tesperat forelsket i ham om Mandagen ; men silen han ikke vil have mig , men De derimod onsscr at eie min Peren , saa er jeg Dem saare forbunden for Deres Artighet og meget beredvillig til at glemme Alt , hvad der tidligere er passent , samt tilsrerge Dem cn evig <b>Kjerlighed</b> , ligesom jeg for har tilsvoret

<h4>1853</h4>Brevet blev overlevere ! Constance paa hendes eget Verelse , hvortil hun havde tåget sin Tilflugt ester Fihosborncs Bortgang og hvor hun siden var forbleien i en Sindsstemning , der paa ingen Maade lunde kaldes misundelsesverdig . Ved en forunderlig Vrangvillighed af Skjebnen stod hun i sine egne <b>Oine</b> ovcrbeviist netop om den Art af <b>qvindclig</b> Vantelmot , som hun af alle Arter nente den oprigtigste Affty for . Det var ittc Vankclmod i Forsetter, i Luner eller i Smag , men det var tusinde Gange vern end nogen af disse Dele , thi dc ! var Hjertets Vankelmodighet » , idetmindste forekom det hende <b>selv</b> saaledes ; thi under den samvittighcdsfultc Sclvprovclses Strenghet , hvilken hun unden lastcde sit eget leg , faldt det hende aldrig ind , at det Feilgnb , hun havde gjort mellem <b>Kjerlighed</b> og Forfengelighet , Intet havtc at gjore mcd hcndcs <b>Hjerte</b> , og at Henry Mortimcr og hans Vers

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008082012002'target=_>Christie, Johan Koren, 1856, Æsthetiske Smaablomster fra ulthima Thule</a><h4>1430</h4>rct begjerlig efter at begyndc paa min Sorg foråt astcegge den med den udsiidte Ssrgekjole ? O , De forstaar Dem godt paa <b>kvindelig</b> Trofasthed og den <b>Kjcerlighcd</b> , som lever hoiere , end i Kjodet , renere end i Sandscrne ? Icg stulde opgive Haabet og vinde derved ? Stakkels Mand , hvor De er fremmed for , hvad et <b>sandt</b> elstende <b>Hjcerte</b> treenger til og vil fole ! Haabet stal leve , indtil den hitre Virkelighed overbeviser mig om , at det er falskt . Og da stal Sorgen vcere mig kjcerere , end Alt , hvad Verden har at byde af , hvad De kalder „ Livct og Glcedcn " . <b>Kjcerligheden</b> har kun en Gjenstand ; den er dens Haab , dens Glcede , dens Liv , dens Alt ; den er det evigt levende Billcde , der staar for Sjcelen uden at kunne udstettcs af nogcn Magt . Lad Gjenstanden <b>selv</b> ds ! ved Villedet er KjErlighedcn rig , og ncest dens Inbcl og Fryd hos den Elstede er Intet den af Vcerd , uden dens Dvalen ved sit Tab og dens Taarcr . leg er endnu rig , min Herre ! ti jcg forstaar den rene <b>Kjcerligheds</b> Fslelse; De er fattig ; ti De kjender den ikke og lester med falske , dedaarende Sosismer og en Visdom af lutter Sandselighed uden Gehalt . Og med dem og den vil De trsste mig ? Gaa ! De er for ulykkelig til at kunne byde mig nogen . "

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014121208023'target=_>Rosenius, C.O., 1868, De ti Guds Bud</a><h4>40</h4>saaledes , at vi virkelig troede og saa , at vi faa Lov at gjore Gud Tjenester, saa stulde vi vist give Alt , hvad vi eiede , for en eneste Anledning dertil , saa at vi siden kunde mindes det og sige , at vi havde faaet gjore Gud <b>selv</b> en Tjeneste . Vi skulde aldrig kunne glemme en saadan naadig Stund , men tale derom , saa lcrnge vi levede . Og for det Andet , hvilken uudsigelig Naade , at Gud saa tudeligt har sagt os , Kvad der er hans Villie , sagt os . hvilke Gjerninger han vil have af os at han ikke har overgivet os til vore egne morke og daarlige Tanter derom . I hvor mangfoldige Daarstaber komme ikke de , som enten ikke eie , eller ogsaa ilke forståa at agte paa Herrens Bud . For dog ikke at tale om Hedningerne, som gjore netop de grueligste Synder til en Gudstjeneste , saa kast blot et <b>Blik</b> paa den Mangde af overtroiske Slikke , som oves i den sande Guds Navn , hvor man forglemmer , at hans egne Bud maa vcrre den sikreste Rettesnor for , hvad der behager ham idet den Gne tror at behage Gud ved at se meget bedrovet ud eller pidsle sig ; en Anden ved et vift Antal Gange at gjeutage en og samme Von ; en Tredie ved at han , idet han kanske forlader en forsvarslos Fader eller <b>Moder</b> , indeslutter sig i et scrreget Rum for at leve alene for Gud o . s . v. Alt Saadant er opkommet deras , at man har vendt sig fra Guds Bud til sit eget formorkede <b>Hjertes</b> Meninger og Folclser . Naar vi bctcrnke alt dette , saa sinde vi , at vi aldrig her i Tiden nok kunne lakke Gud sor den store Naade , at han har givet os sine Vud og lader os faa vide , hvad han vi ! , vi stulle gjore cg derhos , at han har sagt os det i en saa kort og tydelig Form , som i disse ti Bud : thi hvis vi rigtig forståa dem , saa finde vi blot i disse ti Bud al Guds Villie efter Loven , som Christus sagde om Summen af de tvende Lovens Tavler, nemlig Kærligheden til Gud og <b>Kjcerligheden</b> til Ncrsten : . . Af disse to Bud hamger al Loven og Profeterne " . Tcrnk . hvilken Fordel for os . at Herren , vor Gud . har sammenfattet al sin Villie i saadan Korthed , at vi i disse ti Bud have hele hans Lov . saa vi ikke behove at fare vidt ud med vore Tanker , for at faa vide Guds Villie , men finde Alt paa eet Sled . Hvis vi nu komme i Tvivs . om dette og hint er Synd eller en Gud behagelig Gjerning , saa burde vi dog vcrre saa forstandige , at vi tecnkte : Intet kan vcrre sikrere end det , Gud <b>selv</b> har sagt ; Inlet kan vcrre en saa god Gjerning som den , Gud <b>selv</b> har befalet , og Inlet kan vcrre Synd uden det . Gud har forbudet thi Synd er blot del , som strider imod Loven ( se Grundlerten i 1 Joh . 3. 4 ) . Vi burde ogsaa af de utallige Trempler paa vclmenle . men selvvalgte Gudsdyrkelser , som man troede at gjore i Jesu Navn , sorstaa . at i « åndelige Ting er hele vor Natur lutler Merke , at vi derfor maa sperge

<h4>420</h4>at jeg kan sige om Alt , hvad jeg gjor : „ Dette har Gud <b>selv</b> befalet , dette behager ham vel , det ved jeg forvist ? " Endelig siger Luther , at til ret at hcedre Fader og <b>Moder</b> lMer ogsaa : „ at elske dem og > have dem for <b>Oic</b> " . <b>Kjcerligheden</b> gjor Alting let , men hvor Kærligheden fattes , der gaar Alt uvilligt og tragt til . Saaledes ogsaa med Hensyn til det at hadre Foraldrene . Den tvungne Wrbpdighed , den tvungne Tjenestebevisning og den tvungne Lydighed ere blot Skindyder , dpde Gjerninger , i hvilke der fattes Liv og Aand . Forst naar den frie <b>Kjcerlighed</b> ligger til Grund for disse Gjerninger , ere de fulde af Aand og Liv og behage baade Gud og Mennesker . Derfor kan det fjerde Bud , saavel som de Pvrige , ikke opfyldes uden Kærlighed , ftrst til Gud og derncest til Foraldrene . Det er <b>sandt</b> , at disse kunne i Meget og Mangt vare mindre elstvardige , og der kan sindes Meget hos dem , fom fortjener at hades , i Stedet for at elskes ; men de f elv bor dog elskes af os , elskes sardeles derfor , at de ester Guds Bud og Anordning ere vore Foraldre . Med denne <b>Kjcerlighed</b> , ikke blot i Ord , men og ' aa i Gjerning , mener Herren det saa alvorligt , at han endog synes i visse Tilfalde at satte den over <b>Kjcerligheden</b> til ham <b>selv</b> , idet han fordrer , at vi stulle gjore vore Foraldre godt med vort timelige Gods , inden vi ofre ham nogen Gave deraf , og laster det modfattc Forhold . Han siger nemlig : „ Moses haver fagt : crr din Fader og din Moder; og : hvo , som bander Fader eller <b>Moder</b> , stal visfeligen do . Men I sige : naar Nogen siger ti ! Fader eller <b>Moder</b> : det , som du af miq stulde varet hjulpet med , er en Eorban , det er : en Gave til Templet , saa tilstede I h ° m ikke ydermere at gjore sin Fader eller <b>Moder</b> nogen Hjalp . Og I gjore Guds Ord til Intet formedelst Eders Anordning " . Her ser du , hvorledes Herren vil , at du i Liv og Gjerning

<h4>447</h4>gaar som oftest saa med dette Vud , at man ikte betcenker dets Vigtighed , fprend naar man <b>selv</b> er bleven Fader eller <b>Moder</b> og har begyndt at hoste de bitre Frugter af sine egne Varndomssynder . Jo , det er <b>sandt</b> , ogsaa et andet Tidspunkt pleier at vcekke os til bittert at ftle , hvad vi have syndet mod Fader og <b>Moder</b> nemlig en liden Stund efterat de have udaandet sit sidste Suk i Jammerdalen en liden Stund efterat det mod os kjwrligste <b>Hjerte</b> paa Jorden har standset , naar vor eneste jordiske Fader eller <b>Moder</b> ikke mere er i Live ikke mere kan hore vor Bon om Tilgivelse ikke mere tan svare paa vort ommeste Sporgsmaal naar man gjerne midt om Natten vil grave dem op af Jorden , for blot en eneste Stund endnu at faa tale med dem . Fsr var det maaske ikke saa magtpaaliggende , i barnlig <b>Kjcrrlighed</b> at underholde sig med en gammel Fader eller <b>Moder</b> , da den muntre Ungdom behagede os mere ; men nu nu skulde Barnet gjerne tale med fin Fader eller <b>Moder</b> og gjore dem nogen Glcede ; ja nu nu er det for scnt ; uu kan du ikke et eneste j2ieblik lcengere det . O , at vi vare kloge ! O , at vi itide betcenktc os ! Guds Ord taler sagte og mildt , men det lader sig itkc spotte . Man skal rette sig efter det itide , eller og bittert angre det .

<h4>511</h4>Sandhed og Retfærdighed , som Luther kalder „ en til Vrede optcendt <b>Kjcerlighed</b> " , naar det er blot paa Ncrstens Vel og Guds Wre , du har dine Dine rettede , og din Nidkjcerhed vender sig blot mod det Syndige , Falske og Farlige , ikke mod Personen . Paa saadan hellig Vrede have vi Exempel hos den Herre Christus <b>selv</b> og alle hans fornemste Tjenere ; , og hvem skulde ellers kunne rpgte sit Embede som Lcerer , Dommer , Fader , <b>Moder</b> o . s . v. , hvis der ikke fandtes nogen hellig Nidkjcerhed , som iforer sig Vredens Skikkelse ? Ja , huer enkelt Christen kan ogsaa optamdes af hellig Nidkjcerhed for Menneskers Vel og Guds LEre . Kun forglem ikke , at Gud har Dine til at gjennemstue hele dit <b>Hjerte</b> og enhver Bevcegelse deri . Han fer , om det virkelig er denne <b>Kjcerlighedens</b> Nidkjcerhed for Ncrstens Vel , som driver dig , og da en Nidkjcerhed blot imod dette Syndige og Skadelige , som du ser , ikke imod selve Personen . Vil du vide , om det er ren og hellig Nidkjcerhed , som driver dig , saa lcrg Haandcn paa dit . <b>Hjerte</b> , eller strcek den op mod det altseende hellige Vidne i Himmelen , og sporg dig for hans alvidende <b>Blik</b> , om det er nogen fccregen Omstcendighed , som har bevirket , at du har faaet et avindsygt <b>Die</b> til den og den Person , saa at Alt , hvad han saa siden taler eller gjor , kan ikke ret behage dig , ja , at du ligefrem soger at sinde Feil hos ham . Tcenk , om du da sinder eller ved med dig <b>selv</b> , at den forste og egentlige Grund til din Folclse af Uvillie og Nag imod ham var den , at han den og den Gang forncermede dig , at han f . Ex . ikke viste dig tilborlig Agtelse , men tvertimod syntes at foragte dig ; eller at han kanske hnr bagtalt dig og sogt at svcrrte dig blandt Mennesker ; eller kanske har sagt dig dine Feil , givet dig en eller anden Paamindelse , som var saa <b>sand</b> og trceffende , at den just derfor blev bidende ; eller lad os fe ! kanske du itkc kan taalc ham blot derfor ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014052708190'target=_>Hagerup, Augusta, 1862, Grossererens Familie</a><h4>910</h4>Forklaringer end at hore dem af hans egen Mnnd ; thi nu var hun fri for den Uro og Forvirring , som i hans Ncervcerelse opfyldte hende , og til sig <b>selv</b> turde hun sige , at hans Tilstaaelse inderlig havde glcedet hende , havde opvakt Anelsen om en Lykke , der bragte hendes Kinder til at rodme , hendes <b>Hjerte</b> til at banke stcerkere og et straalende Smil til at spille om hendes Lceber . Men for Halfdan maatte dette endnu vcere en dyb Hemmelighed , hun maatte forst prove ham , forst blive overtydet om , at hans <b>Kjcerlighed</b> virkelig var <b>sand</b> og standhaftig , og indtil enhver Tvivl herom var forsvunden, skulde hun nok vise ham , at hun ikke var saa let at gjenncmstue , som han engang havde paastaact ; men at hun lige saa godt som nogen anden <b>Kvinde</b> kunde dslge sine Fslelser , naar hendes <b>kvindelige</b> Veerdighed gjorde hende det til Pligt .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006103000001'target=_>Halling, H., 1853, liden Postille for Børn</a><h4>1199</h4>Nedste , da koster du din Fader og <b>Moder</b> Sorg og Taarer , og disse Taarer ere dyre — og stal engang hvile tungt paa din Samvittighed ; men ogsaa Jesus grader over dig — og hvor dyre ere ikke Hans Taarer ; hvor ville ikke de engang brande paa din Samvittighet » ! Du ved , at Jesus elsker alle Mennesker og har af <b>Kjcerlighed</b> til dem ostret sig <b>selv</b> . Men Ingen elsker Han hoiere end Varnet . Odu er Ham saa kjcer . saa dyrebar ! Men naar du , Hans kjerre Barn , forladerHam ; naar du glemmer Ham og lader dig drage med af Verdens store Strsm , der forer " til Fortabelsen. — o hvor bitfte Taarer flyde ei da af Jesu <b>Die</b> ; hvor udbreder den trofaste , smme Frelser da sine Arme efter dig , og beder dig saa inderlig betcenke , hvad du gjsr , betamke , hvem det er , du forlader , og komme til Ham igjen , komme medens det endnu er Tid , komme , fsr det er forsent! O , maatte Jesu Taarer rsre vort <b>Hjerte</b> dybt , kjerre Bsrn ! Maatte vi ved dem bringes til at groede Angerens Taarer over os <b>selv</b> , over at vi saa ofte have bedrevet Ham ved vor Ulydighet » , — maatte

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014031808034'target=_>Riddervold, Jul., 1870, Johannes's Evangelium</a><h4>707</h4>Naudbede : „ Abba , Fader / forst da tilbede vi i Aand og <b>Sandhed</b> . „ Thi og Faderen soger faadanne " , osv. her kalder Jesus indtrængende paa <b>Kvindens</b> <b>Hjerte</b> ; her staar han sor hende , den Guds Udsendte , i hvem den nye Tid var oprunden , og som sogte for Faderen sande Tilbedere ved at kalde og drage til fig. Til hvilken ydmyg Tak maatte ikte disse Ord stemme <b>Kvindens</b> <b>Hjerte</b> , hvem Guds <b>Kjcerlighed</b> idag saa uventet for hende havde opsogt for at give hende Fred . Faderen ssger ja , hvem maa ikte dette Ord fylde med Tak og Pris ? Guds dragende <b>Kjcerlighed</b> opsoger os paa vore vildfarende Veie ; kom Me han forst tii os , kom vi aldrig til ham , thi vi har forvendt vor Vei . Tilbedes i Aand og <b>Sandhed</b> vil Gud , thi han er <b>selv</b> Aand , hvorved betegnes hans evige , personlige Herlighedsvcesen ; han er den evige , almcegtige og hellige Gud , hvem Himlene og Himlenes Himle ilke kunne rumme ( 1 Kong . 8 , 27 ) . Ham stulle vi tjene i Hjertets ydmyge Gjenkjcerlighed , og i hjertelig Offerlydighed . Hoor dette mangler , er vor Tilbedelse kun et Kainsoffer , og det ligger dybt i vor Natur at ville ligesom fordum Isderne affcerdige Herren med blot udvortes Offrc og dermed ligesom kjsbe os Guds Velbehag . Hvo hender Me det pharisceifle <b>Hjertes</b> Snigveie , for at slippe Tilbedelse i Aanden , som er at opgive alt sit Eget ? Gud er en Aand det staa levende for os , nåar vi ville dyrke vor Gud , som en Dom over alt Hykleri og Skin . V . 25. 26. <b>Kvinden</b> siger ti ! ham : Jeg ved , at Messias kommer hoilkct betyder Christus ; uanr hau kommer , skal han forkynde os alle Ting . Icsns siger til hcndc : Icg er (Messias), som taler med dig .

<h4>710</h4>Kvindens <b>Hjerte</b> tsrster efter Lys og <b>Sandhed</b> , og hnn haaber , at hun stal faa sin Lcengsel stillet , naar den Time lommer , om hvilken Frelseren havde talt . Samaritanerne ventede en Messias efter 5 Mos . 18,18 . , hvem de kaldte med et Navn , der betyder „ Omvenderen " , eller „ den der kommer igjen " , idet de tcenkte at Moses <b>selv</b> flulde opstaa igjen . Saare ringe Tanker havde de om Messias, i hvem de kun ventede en oplyst Laerer , men saa havde de heller ikke som Isderne de mange falste Tanker om ham som en verdslig Konge med verdslig Glands vg Magt . Derfor behovede ikke Frelseren her at frygte for , at det flulde misforstaaes , naar han kaldte sig Messias, og da <b>Kvindens</b> <b>Hjerte</b> suttede efter Fred , kunde hans <b>Kjcerlighed</b> ikte lcengere stjule sig , og han sagde ligefrem : „ Jeg er Messias " . Saaledes havde han endnu ikke givet sig tilkjende for Isderne ( se Matth . 16 , 20. o . a . St . ) . Nu havde <b>Kvinden</b> faaet Svaret vaa Hjertets dybeste Sporgsmaal , nu havde hun faaet en Smag af det levende Vand . Ikte alene <b>selv</b> var hun fort til Troen , men „ hun blev og saa et Troesbud for Andre " . I glad Taknemmelighed iler hun bort for at bekjende sin Frelser .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014082608012'target=_>Vibe, Johan, 1875, Alexander Møllers Erindringer</a><h4>1932</h4>Hverken Presten Hsrby eller Ungdommen kunde blive troet af at udmale , hvor hyggeligt de havde havt det paa Turen , og hvor udmoerket de havde moret sig i de tre sidste Dage . Fem Minutter efter kom Georga ind med Theen , og vi satte os tilbords . Grossereren trakterede med Vin , som erholdtes taalelig god fra Landhandleren i Noerheden, og der herskede den hjerteligste Stemning . Georga gjorde dog en Undtagelse . Hun var cengstelig og aandsfravcerende ; men mod mig var hun hele Tiden Forekommenheden <b>selv</b> . Hun bsd mig The fsrend de Andre og gav mig den stsrste Kop ; hun spurgte , om jeg frss , og da jeg et Meblik hvilede min Kind paa Haanden og tilkastede hende et vemodigt <b>Blik</b> , spurgte hun mig med <b>sand</b> <b>kvindelig</b> Deltagelse , om jeg havde Tandpine . Hun misforstod mine Fslelser , men var hendes Spsrgsmaal derfor mindre <b>kjcerligt</b> ?

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011051310001'target=_>Glümer, Claire von, 1880, Dønninghansen [i.e. Dønninghausen]</a><h4>444</h4>« leg vil ingen Ting negte . . . jeg ssger kun at forklare for dig — og egentlig ogsaa for mig <b>selv</b> , hvad der foregaar i mig " , sagde han . ~ Du har vist hsrt Klagemaal over Mcendenes Forfcengelighed — men du har aldrig hsrt nok , det kan du tro mig paa ! Imod vor bedre Overbevisning, ja imod vort <b>Hjertes</b> Manelser giver den os redningslss i enhver koket <b>Kvindes</b> Maat , naar hun forstaar at smigre denne vor Forfcengelighed . Koketteriet i og for fig er den Smiger , vi ej kan modstaa. Men hvor ofte tsrster vor Sjcel , mens vi henfalder til dette onde Trylleri — og forsmcegter , naar den rette <b>Kjcerlighed</b> ej kommer den til Hjcelp ! " lohannas <b>Hjerte</b> bankede ; men hun vilde lade , som hun ikke forstod og sagde smilende , uden at se op fra fit Arbejde : ~ Den stakkars <b>Kjcerlighed</b> ! — hvordan stal den bcere sig ad , naar den ikke ogsaa forstaar at kokettere ? " ~ Det behsver den ikke ! " svarede Otto heftig . ~ Den behsver kun at vise sig alvorlig og enkel som den er , og alle Heksekunster bliver til Skamme . — Tro mig , Johanna , hvis jeg en Gang fandt den — og en Tone , et <b>Blik</b> vilde aabenbare den for mig — da var jeg reddet for beftandig".

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010021803024'target=_>Bungener, Felix, 1857, Julien</a><h4>390</h4>omtalte Helgeninde , der var dyd i Slntningen af det syttende Aarhundrede , var ikke blot i Ry paa Grund af de Syner og Undergjerninger , som man tillagde hende . Hun havde forfattet en lille Bog , betitlet : „ lesu hellige <b>Hjertes</b> Tilbedelse " ; det var dette lille Skrift , der var bleven benyttet af lesuiterne og der havde givet Anledning til Festen for det hellige <b>Hjertes</b> Tilbedelse , den Fest som Ertebistoppen havde sygt at indfyre i Messebogen og som Regjeringen havde stryget . Dersom Skriftets Forfatterinde hyitideligen erkleredes for Helgeninde , saa var dermed ikke Lidet vundet , og i ethvert Tilfelde erholdt lesuiterne derved en Slags JEresopreisning af Paven , thi det var en velbekjendt Sag , at stjyndt Ordenen var oplyst , saa var det dog denne , der forlangte Canonisationen , og denne , hvem Romerhoffet begunstigede , nåar det indrymmede Begjeringen . Men den Maade hvorpaa Bistoppen af Antuu udviklede sin Plan var i <b>Sandhed</b> ikke mindre besynderlig end Sagen <b>selv</b> . Han vidste heelt vel , at Herr 66 Beaumont ikke vilde cmsee ham for en Nyomvendt , der troede paa Marie Alacoque ' s Helgenmircckler ; han betvivlede endogsaa meget , at Prelaten <b>selv</b> bekymrede sig om den hellige <b>Qvinde</b> undtagen af Bihensyn og i lesuiternes Interesse . Han gav det da heller ikke Udseende af at han var en Helgcntilbeder , men dog var der heller ikke nogetsomhelst Spor af Tvivl eller Ironi i hans Stemme eller <b>Blik</b> . Det gil her som nåar en Skuespiller henriver En , stjyndt man i samme Vieblik siger sig <b>selv</b> , at man er Vidne til et Blendverk; det var den samme Mant » , der engang skulde fremkomme med den Paastand , eller hvem man tillagde den Paastand — thi det kommer her nd paa Eet — at „ Talen blev givet Menneskene til et Skjul for deres Tanker . "

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016030829005'target=_>Gislesen, Henriette, 1851, Helligaftenerne</a><h4>139</h4>Moderen . For at frie os fra Sorg og Taarcr , ikke alene i dette Liv , men endnu mere i det Tilkommende , mit dyrebare Barn , er det at vi have modtaget Befalingen om at gjore alt i Jesu Navn . Hvad vi af et fromt , troende <b>Hjerte</b> i <b>Sandhed</b> gjore eller sige deri , fremkalder aldrig Angerens Taarer , netop fordi Navnet , for et saadant <b>Hjerte</b> ikke kan andet , end blive til en hellig Provesteen for Gjerningen og Ordet <b>selv</b> , — til en hellig Paamindelse om at lade Ord og Gjerning ste efter Guds Villie isse efter dets egen . Troer ikke , mine Born , at disse Ting , om hvilke vi nu tale , cre for ringe og uanselige til at vi saa alvorligen skulde gaa ind paa dem . Ingen Ting er i Virkeligheden ringe , <b>selv</b> ikke hos det mindste Barn , naar det er et Udtryk for Hjertets Folelse eller for Sjcrlcns Tilstand , om denne Tilstand end , ifolge sin Natnr , kun hos Barnet kan afprcege sig i det som synes Ringe og Übetydeligt for det ydre <b>Blik</b> ; thi det er stedse kun paa Hjertet at Gud seer . Jeg stal fortcrlle Eder et Eremvel paa hvor alvorlig bekymret og nidkja-r Herren er over sk Navns Wre , <b>selv</b> der hvor det gjcrlder de i menneskelige Oinc übetydeligste Ting . Jeg vil derved atter komme til at omtale for Eder en Karakteer , elskeligere end de fleste Andre jeg har kjendt , og med hvilken I allerede , fra en af vore forrige Helligaftener , have gjort Bekjendtstab . Anna . Hiin lille afdode Lydias <b>Moder</b> , mener du sikkert .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016111648038'target=_>Payn, James, 1878, Arthur Conway</a><h4>201</h4>bur Menneskelige Fslelser , " svarede Raymond med Bitterhed. „ Det er din <b>Moder</b> , som har sat dig saaledes oft mod ham . " „ Jeg er aldeles ikke opsat mod ham , Raymond , men bestemt vaa at se Tingene , som de i Virkeligheden er . " „ Tet er altsaa din Mening at fortælle mig , at du negter mig at sige ham , at du har samtykket iat blive min <b>Hustru</b> ? " „ Det er det ja ! " „ Du almægtige Gud I Det vil altsaa med andre Ord sige , at du ikke elsker mig ? " raabte den unge Mand , idet han for tilbage som ramt af et haardt Slag , samtidig med at Ansigtet blev overtrukket med Fortvivlelsens Bleghed , og hans Ojne sendte hende et <b>Blik</b> , der lignede det , en tro Hund kaster vaa sin Herre , naar han staar den . Og alligevel var det ikke i nogen medlidende , men hellere indigneret Tone , at den unge Pige svarede : „ Det er grusomt af dig , Raymond , at pine mig saaledes ! Hvad kan du dog vinde ved at vriste en saadcm Tilstå aelse fra mig ? Eller fynes du , det er mandigt at behandle en <b>Kvinde</b> , der aldrig kan blive din <b>Hustru</b> , vaa en saadan Maade ? Er det maaste for at kunne prale af , at jeg ogsaa var en af de mange uuge Piger , du kunde faaet tilcegte , om du havde villet ? " „ O , Nelly , Nelly , hvem er det , som nn er grusom ? " „ Jeg er bedrovet , snare bedrovet , Raymond , og har slet ikke til Hensigt at være grusom . Vien der gives Ting , som — med andre Ord , du førstaar ikke , hvad en ung Piges <b>Hjerte</b> er . Nej , jeg kan ikke tillade dig at fortælle , at jeg er gaaet ind vaa at blive din <b>Hustru</b> , <b>selv</b> om du derved skulde kunne bevæge din Faders <b>Hjerte</b> . Vi Conwayer er meget fattige , men ogsaa meget stolte . Jeg har ogsaa en Fader , der ikke er rig som din , men , hvis gode Mening om mig jeg sætter nhyrc megen Pris » paa , og jeg kjender hans Anskuelse med Hensyn til det , vi nys har talt om . Det er <b>sandt</b> , at han og Hr. Penni- > uick er gode Venner ; men det er et ulige Venskab , ikky

<h4>690</h4>Fslelse hos Dem . Forekommer det Dem maaske besynderligt, at jeg endnu ikke har bessgt Dem , kan jeg kun sige , at legemligt Ildebefindende er Grunden dertil . Jeg sendte Raymond hen for at forklare — " „ Ralph Pennicuick , " afbrsd Fru Conway med hces Stemme ; „ jeg er kommen hid for at faa <b>Sandheden</b> at vide ! " „ <b>Sandheden</b> , min kjære Frue ? " „ Ja , <b>Sandheden</b> om min Mand . " Pennicuick vred sig under hendes gjennemtrcengende <b>Blik</b> ; men han svarede ikkedestomindre roligt nok : „ Hvad Raymond har fortalt , hørte han af mig <b>selv</b> , og jeg kan derfor kun gjentage — " „ Det er derfor , jeg ikke kan tro det , " afbrsd Fru Conway heftigt . „ Det er Deres Munds Udsagn , jeg tvivler vaa , fordi den er vant til at lyve . Lsj De kanske ikke for ham , medens han levede ? Hvad ! Tror De kanske , jeg har glemt , hvorledes De , dengang Deres Raymond kun var nogle Maaneder gammel og jeg en anden <b>Moder</b> for ham , medens min Mand var Deres bedste Ven , hvorledes De da satte alle disse hellige Baand ud af Betragtning og havde den Frcekhed at tilbyde mig Deres <b>Kjærlighed</b> ? " „ Men dette er jo nu faa lang Tid siden , kjære Frue , " svarede Pennicuick med hces Latter . „ Kan nok være ; men jeg har dog ikke glemt det og er sikker vaa , at den Mand , der var istand til at forraade Venskabet og bortkaste Erindringen om sin nys afdode unge Brud , ikke har forandret sig meget siden den Tid . Der findes sikkerlig ikke Offer stort nok , <b>selv</b> ikke Deres egen Sons Liv . som De tog i Betænkning at modtage , om det kun gjaldt Tilfredsstillelsen af et eneste Dnste af Dcm . " „ Jeg ved , De altid har været et Monster vaa Dyd , Frue , medens jeg , naa , De kan jo lænke om mig , hvad De vil i den Henseende ; men at rippe op i Sager , som er en hel Levetid gamle , iscer da de ingen Fslger havde — " „ Der tager De Fejl ! ThiFslgen var , at jeg mistede min Mands <b>Kjærlighed</b> ! "

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015061048110'target=_>Rellstab, Ludwig, 1835, 1812</a><h4>324</h4>, gjennemtr.rngt af den rcrdsomme uimodsigelige <b>Sandhed</b> hun er tabt for Dig ! Det matte <b>Die</b> , den blege Lwbc lod hende meget mere , end hendes stumme Hilsen , Hendes Mangel paa Kraft til at tale , bem « rke , at den venlige <b>Moder</b> ikke lignede sig <b>selv</b> mere . Hendes <b>Hjerte</b> trak sig krampagtigt sammen ved Bersrelsen af denne nye Elendighed , der kom over hende ; men hun beholdt sin Kraft , og hendes Mund smilte imedens hendes Bryst blev sonderrcvct af navnlos Smerte .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015020408003'target=_>Munch, A. $q Andreas, 1856, Billeder fra Nord og Syd</a><h4>911</h4>Som Folge heraf levede ogsaa min Kunst oft igjen i min <b>Kjcerlighed</b> , og udviklede sig i den . Skjont jeg ikke tcenkte Verpaa , skjont mit hele Veesen forekom mig udelukkende opfyldt af min <b>Kjcerligheds</b> Gjenstand , voxede min kunstneriske Evne dog stille frem i dens Varme , og inden jeg kunde gjore mig <b>selv</b> Regnskab derfor , var min Haand bleven let , mit <b>Oie</b> sikkert , min Tanke klar , min Phantasi frugtbar , og mit fulde <b>Hjerte</b> gav <b>Sandhed</b> og Foleise til det Alt . Legende forst , som for Tidsfordriv i Ventetimerne mellem hver Gang jeg saa hende , henkastede jeg Skitser ; jeg angav Comftositioner , hvis Skjonhed og Dristighed forbausede mig <b>selv</b> , og til hvis Lige jeg sor med alt mit Studium og Arbeide ikke havde kunnet narme mig . Engang bleven opmoerksom ftaa denne forogede Evne , begyndte jeg atter at elske min Kunst og drive alvorligt paa den , fordi den sprang ud fra min <b>Kjcerlighed</b> , og igjen fiulde gaae tilbage til denne . At gjore Fremstillingen af det Skjonne og Ophoiede til sit Kald , derved at vinde Navn , Formue og Berommelse , forekom mig nu vel veerdt et Livs hoieste Anstrengelse , naar jeg kunde bringe det Alt som et Offer til den Elskede . Og Lorenza , var huu istand til at dele et sandant Liv , ti ! at fatte en saadan Hyldest ? Hnn Var ikke institutdannet , kunde ikke tale fiint om Wsthetik ' og Kunst , huu var , som de fleste romerske <b>Damer</b> , <b>selv</b> af de hoiere Stcender , endog hvad man hos os vilde kalde uvidende, men hun hnvde modtaget en Skjonhedssands af den milde Natur , rigere og fristere end hine tilleerte og tilsnorede Damers grundlose Flostulercu , hendes hele Vcesen var

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011060704032'target=_>Feragen, A.M., 1875, Hjem og Skole</a><h4>406</h4>forenet med Stjonsomhed . Villien hos stige aandskraftige Born stal netop itte knektes , thi dette vere det samme som at myrde dem aandeligt , men den stal boi es og led es , nemlig bores under Guds Villie og ledes bort sra alt det , som er ondt , til det , som er godt , <b>sandt</b> og ret . En saadan boiet og ledet kraftig Villie er en uvurderlig Skat at tåge med ud i Livet ; den har Mod og Magt til at staa fast med et bestemt Nei i Fristelsens Stund , Mod og Magt til utretteligt at kjempe for den gode Sag , trods Modstand , Spot og Hann . Men hvad kan vel en knettet Villie udrette for sig <b>selv</b> og andre ? Intet , fordi den er som et Vrag , der tumles om af alslags Veir og Bolgegang . Ver derfor isse mismodig , du Fader og <b>Moder</b> , forn har faar et eller flere Born , udstyret af Naturen med en Villie , der giver sig tilkjende som Stivsind og med overstrommende Folelser , men tak eders Gud derfor og bed ham om Visdom til at fore dem paa ret Vei . Det gjelder om tidligt at lere dem Forsijel paa godt og ondt , at vette hos dem <b>Kjerlighed</b> og Hoiagtelse for det forste , men Had og Assty til det sidste , og i dette Diemed ftittigt minde dem om , at de er dobte til at . . forsage Djevelen med alle hans Gjerninger og alt hans Vesen". Saa maa de lere at kjende sig <b>selv</b> og faa <b>Die</b> paa , at der rorer sig saa meget hos dem , som maa forsage.s , og at de maa sutte til Gud om Kraft til at gjore dette . Naar saadan Veiledning og Tugt fra Foreldres og Lereres Side ster i <b>Kjerlighed</b> forenet med Fasthed og Bestemthed , saa vil Barnet

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111401049'target=_>Arentzen, Kristian, 1879, Adam Oehlenschlæger</a><h4>3049</h4>Et Par Maaneder efter , at Oehlenschlåger havde udgivet den smukke Saga , hvormed han vendte tilbage til sin første poetiske <b>Kjærlighed</b> , døde i Sommeren 1841 faa Uger før hendes 59 aarige Fødselsdag hans Ungdomselskede , Hustruen , der i en Menneskealder  trofast med 8 ^ 931 - 1 H61i66 <b>Blik</b> , vaagende , værgende , kæmpende , aldrig hvilende " ( som det tl-k6K6H ( i6 116^61-i C . Hauchs ypperlige Mindedigt over hende ) havde staaet ved Digter ens Side . En fattig <b>Kone</b> , der undertiden hjalp i Huset med Vadsk og Andet , var bleven syg ; den brave , altid hjælpsomme Fru Oehlenschlåger med sit  Martha-Værk " og med sit  ømme <b>Hjerte</b> " ( for at bruge Udtryk af Vilhelm Birkedal om Digterens <b>Hustru</b> ) , der i de sidste Aar <b>selv</b> havde været lidende , besøgte hende og fik Tyfus . Oehlenschlåger skrev dengang paa  Dina " ; og Scenen henimod Slutningen af dette tragiske Drama , hvor den miskjendte Korfits Ulfeld og hans trofaste <b>Hustru</b> drage bort fra Fædrelandet , var det Bidßt6 , DiFt6r6n læste for den syge <b>Hustru</b> . XrißtiaQ6B smukke store blaa SiQ6 lvidtes

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009072703015'target=_>Heiberg, J.L., 1862, J.L. Heibergs samlede Skrifter</a><h4>94</h4>have sagt ham , at Charlotte endnu var übekjendt med saa hceftige Lidensiaber , og at deres utvetydige Ittrmg i store Selstaber , for Alles Mue , ikke kunde Andet end scette den stakkels Pige i den ststrste Forlegeuhed. For Enhver , som kjendte hende npiere , og iscerdeleshed for mig , som hendes Fcetter , der havde seet og kjendt hevde fra heudes spcedeste Barndom as , var det tydeligt , at Carls Paatrcengenhcd i hpi Grad geneerte hende , og at huu var saa langt fra at gjengjelde hans <b>Kjcerlighed</b> , at hun tvertimod saae den med Mishag . Carl <b>selv</b> kunde ikke undgaae at have en Fplelse deraf , men denne bidrog kun til endnu at forhpie hans Lidenskab . I Selskabet havde han ikke Vie for Andet end Charlotte ; han talte kun med hende , hans <b>Blik</b> var kuv henvendt vaa hende . Naar Charlotte , forlegen over den Opmcerksomhed , som Carls Ovfprsel vakte i Selskabet , flyttcde sig hen ftaa en anden Plads , saa vårede det kuv et Meblik , og han fulgte efter hende . Gav hun sig i Samtale med En af de tilstedeværende <b>Damer</b> , saa blandede Carl sig pieblikkelig deri . Denne Adfcerd , som i ethvert Huus vilde vcere upassende , maatte dobbelt stpde hos min Onkel , hvis Frue endnu stedse ansaae sig for eneberettiget til at modtage ethvert Offer vaa Galanteriets Alter . Man beskylder hende for at have havt mange Elskere . Hvorvidt det er <b>sandt</b> , skal jeg lade vcere usagt ; ' ' meu saa Meget er vist , at hun gjpr

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010021803030'target=_>Jomtou, Johan Jørgensen, 1858, Caroline Mathildes Livshistorie</a><h4>237</h4>egtc . Den Bevidstbeb , at have med Trossal ? opfyldt min Skyldighet » imod Hs . Majestet , og den Agtelse , jeg stylter mig <b>selv</b> , sial hverken Ondslab eller Avind berove mig . " Imedens hendes stygiige Gemahl var frnverende residente hun pna Frederiksberg , og hendes Opforsel var daddelfri . Endstjondt hun var smigret og truet af forskjellige Factioner , forenede hun sig dog ikke med Nogen eller viste mindste Tilboielighed til Politik . Hun syntes at fole en <b>sand</b> <b>moderlig</b> <b>Kjerlighed</b> for sit Varn , og , nåar man klagede over , ikke at tunne styre det , eller frygtede for , at det ikke befcmdt stg vel , saa sov baade Ammen og Varnet i hendes Verelse . Hun besogte undertiden , og blev besogl af Enkedronningen , som boede meget eensomt . Hun var voren stor , og syntes meer <b>qvindelig</b> end da hun kom til Danmark . Den kraftige Sundhet » glodede paa hendes Kinder . Hun gav ofte sti Varn <b>Die</b> . og en mere interessant Gjenstand kunde neppe tenkes end at see denne elskverdige livfulle Dronning lege med sit Pattebarn . I denne Periode besogte hun Forpagternes og Vonternes Huse i Nerheden af Slottet , og endstjondt hun ikke kunde lale flytende med disse tatnemmelige Mennesker , vandt lun dog deres Hjerter ved sin Nedlatenhet » , ved at besoge deres Huse , smile til deres Koner og Dottre , og uddele brugbare Foreninger . Saa usiyldigt tilbragte Dronning Mathilde sin Tid . "

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009072703018'target=_>Mau, E., 1865, Johannes Ferdinand Fenger</a><h4>166</h4>ved at gaae op for Alteret og kaste sig ned paa sine Knee , hvor han da under mange , mange Taarer bekjendte Herren sine Overtredelser . Det vårede lenge , vist en Time , inden han kunde rive sig lj ? s fra Kirken og gaae hjem . Hans <b>Kone</b> kom ham impde .  Hvor bliver du af , lpd Spprgsmaalet , og hvorledes seer du ud ? Dine Dine ere rpde og forgredte . "  la , svarede han , det er <b>sandt</b> , jeg er bedrpvet , men det er den Bedrøvelse , som er ester Gud , og den trenger baade jeg og du til . " Nu var Isen brudt , men nu begyndte ogsaa fprst Kampen ret . Stundom fplte han Guds Naade og var blpd ; men saa reiste det gamle Menneske sig i ham og forsagte at kempe for sit Liv , og da kom der en saadan Uro over ham , at hans <b>Kone</b> siden har forsikkret mig , at hun frygtede for hans Forstand . Da tyede hau stundom op i Skolen , stundom til Kirken , for at bede ; men varig Fred kunde han ikke finde ( han forstyrredes iser af onde Tanker ) . Saaledes gik Tiden hen indtil Mandag Aften , da han laae og kunde ikke sove . Da faldt det ham paa eengang ind : hvorfor er du saa urolig ? det er <b>Kjcerlighed</b> , som mangler dig ; giv dig hen til Gud , som er <b>Kjcerlighed</b> og som annammer angergivne Syndere . Da kom Freden til ham ; han begyndte at bede for sig <b>selv</b> , for sine Ncermeste , for sine Venner og for dem , han ikke kunde lide , og sov efter den BM spdelig ind , og var nceste Dag let og glad . Gleden spillede ud af Oinene paa den gamle Mand , han var i Gang og i Bevegelse som en Ungersvend . Og hvor var mit <b>Hjerte</b> glad ! Jeg gav ham alle de beoste

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014090108029'target=_>, 1880, Jung Stillings Liv</a><h4>965</h4>Nu levede altsaa Stilling glad og tilfreds i Marburg. Hans Wgtestab var ham en daglig Kilde til den hsieste Lykke , som kan tomles her i Verden , thi Selma elskede ham af sin egen SM ; hendes <b>Hjerte</b> slog ham uafladelig imode , og naar hans Sind var forfagt , gik alle hendes Bestræbelser ud paa at opmuntre ham og indgyde ham Mod . ' Omgangen med hende har ogsaa dannet ham , saa at han ikke mere stoder an i fint Selskab . For hans Born af forste Mgtestab var hun alt , hvad Faderen kunde snske , hun var <b>Moder</b> og Veninde paa samme Tid . Hvad hendes Udseende angaar , da var hun af Middelshside, havde et aandfuldt Ansigt , og Velvilje og Menneskekærlighed lyste ud af de blaa Bine . <b>Selv</b> i saadanne Ting , der pleier at ligge udenfor de <b>kvindelige</b> Enemarker, havde hun et klart og skarpt <b>Blik</b> , faa at hendes Mand ofte raadfsrte sig med hende og gjorde vel deri . I religiose Sager tcrnkte hun som en oplyst kristen og var opfyldt af varm <b>Kjcrrlighed</b> til Gud og Nersten . Hun var sparsommelig , hvor det gjordes fornsdent , men gavmild og godgjorende paa famme Tid . Hendes Beskedenhed overgik alle Gramser ; hun underordnede sig altid sin Mand , <b>selv</b> naar han handlede efter - hendes Rand ; hun sogte aldrig at glimre , men vandt dog alle for sig , hvorsomhelst hun viste sig . „ Hvem Gud elsker , giver han

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016050229002'target=_>Abbott, John S.C., 1844, Fredens vei</a><h4>258</h4>deeltage , eller soger Du at forstyrre eller hindre hende fra at tage Deel i den Gudstjeneste , som hun anseer for den rette , saa viser Du en ikke mindre Intolerance end den , der stiftede den Spanske Inqvisition og tcrndte Baalet ved Smithfielo . Jeg Milde ncrsten troe , at en ncermere Undersogelse vilde lede til det Resultats at Pavedommets Forfolgelsesaand i Virkeligheden heller ikke er uddod i vort frie Land . Jeg skulde virkelig frygte for at ogsaa hos os mangen <b>Qvinde</b> sukker under religios Tvang og Undertrykkelse . Uden anerkjcndts religiose Rettigheder , maa hun da som sin Mands Slavinde folge ham til den Kirke , som han <b>selv</b> besoger , hvor tungt det end er for hendes Samvittighed , hvor uforeneligt med hendes Folelser . Kan en saadan Mand vcrre besjelet af crgte frisindede Grundscrtninger eller kan hans <b>Hjerte</b> vcrre opfyldt af <b>sand</b> Gudsfrygt , naar han , <b>selv</b> i vore oplyste Tider , kan tillade sig en saadan Vold mod et Medmenneskes aandelige Frihed ? Den , der ei kan taale den religiose Frihed i sin egen Familie, han behovede kun at besidde tilstrækkelig Magt . for ogfaa i Staten at underkue den frie Undersogelsesaand. Den , der kan ville gjore sit Livs Ledsagerinde til et ulykkeligt Offer for religios Forfolgelse , han vilde sikkerligen endnu mindre tage i Vetcrnkning at udove en trykkende Magt over Andres Samvittighed . Fra Familiehjemmet skal Tolerancen ligesom <b>Kjcrrligheden</b> have sit Udspring .

<h4>514</h4>Du undviger Nlt hvad der kan saare deres Folelser , al u anmodet Indblandelse i deres Anliggender , bor Du gribe enhver gunstig Anledning til at bringe dem tilatbetcenke deres Ansvar hisset , den doende Frelsers <b>Kjcerlighed</b> og denHelligaandsalvorligeKaldelse . HarDu forst et <b>Hjerte</b> , der stammer af christelig <b>Kjcerlighed</b> , saa behover Du kun en almindelig god Forstand forat indsee , naar Du bor overtale en Anden til at folge Dig hen til en Gudstjeneste eller til at samles med nogle christelige Venner til Bon i Dit Duus , — naar Du bor laane ham en alvorlig Vog eller <b>selv</b> tale med ham om hans Sjels evige Vel . Med en god sund Forstand og et <b>sandt</b> christeligt Sindelag ville vi ikke saa let tage feil i dette Punkt . Tag Din Nabo i Haanden , naar han ligger paa sin Sygeseng , og henled hans Tanker paa den Frelser , der blsdede paa Korset for hans Synder . Tal til Moderen , der med taarefyldt <b>Oie</b> betragter det syge og lidende Varn , som hun holocr i sine Arme ; tal til hende om den Frelser , der sagde : „ lader de sniaa Vorn komme til mig og formener dem det ikke , — om ham , jom gjennem Vedrovelsen soger at drage hendes <b>Hjerte</b> fra denne Verden til sig .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011051320007'target=_>Charles, Elizabeth, 1880, Frafalden, men ikke fortabt</a><h4>1199</h4>sagde Eucharis ; ~ en stakkels , vildfarende , fristet , falden <b>Hustru</b> , som fristede sin Mand — og som ikkedestomindre var bedre , " tilftiede hun med et pludseligt og Mblikkeligt Udbrud af Vrede , ~ end den Mand , hun fristede : thi han var lav nok til at anklage hende for Gud , hvilket hun aldrig vilde have gjort . Djcevelen er en Anklager saavelsom en Frister . Stakkels , fristede , forladte Eva , hvorledes folte hun sig tilmode , da hun hMe sin Mand anklage hende for Gud ? Skal jeg sige , hvorledes jeg tcenker , hun fplte ? Jeg tomter , at da hun — skjent kun en stakkels , fristet <b>Kvinde</b> — dog skulde blive alt Levendes <b>Moder</b> , lod Gud aldrig <b>Kjcerlighedens</b> Kilde blive udtMret i hendes <b>Hjerte</b> , og medens Adam undsknldte sig <b>selv</b> og anklagede hende for Gud , tcenker jeg , statkels Eva laa nedkastet i Stevet for sin Mands Fadder og for Guds Fadder i sin Ddmygelse og Smerte , itke anklagende nogen anden , men lun sig <b>selv</b> , og undfkyldende ham . Og derfor var det maaske , formoder jeg , at i Evas <b>Hjerte</b> og ikke i hans oprandt ftrst et guddommeligt Haab , idet hun fluttede sin til fit <b>Hjerte</b> og onskede ham velkommen som ~ et Menneske fra Herren " , det Menneske , i hvis Slcegt Menneskens SM tilsidst skulde stjcenkes os . Men husk , " tilftiede hun med et straalende , begeistret <b>Blik</b> i sine ~Menneskens Sf < n , fordi Han var Son af Maria , den Velsignede blandt <b>Kvinder</b> . Og fra den velsignede Marias Lcrber kom det fyrste Svar , som forst ophcevede Forbandelsen . Det var en <b>Kvinde</b> , som — idet hun med tilbedende <b>Hjerte</b> og villig Lydighed underkastede sig Guds Vilje — begyndte at hceve Evas Forbandelse . Eva blev fristet og faldt og fristede <b>selv</b> ; men hun sank aldrig saa dnbt , at hun mistede <b>Kjcrrligheden</b> og blev en Anklagerste . "

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016051048056'target=_>Farrer, Henrietta Louisa, 1863, Alice Beresford</a><h4>547</h4>Georgina havde aldrig Kraft nok til at fore Beviser for sine Paastande , og Bertha begyndte at mumle noget om Prædikener , og Fslgen blev , at begge de unge <b>Damer</b> begav sig bort og lod Broderen i uforstyrret Besiddelse af Spisestuen . Augustus forblev i sin yndlingsstilling foran Kaminen med sin Bog hvilende paa Kamingesimsen ; men det var i sit eget <b>Hjerte</b> og ikke i Europas Historie , at han lcrste , idet han førskede deri paa en Mande , som han oftere havde anvendt siden Alices Tilbagekomst . Han indsaa klart , hvilke Mangler der fandtes i Hjemmet , fslte dybt Savnet af den rette Harmoni og Orden ; men det havde altid voeret hans Sædvane at dvoele ved den Skyld deri , som tilfaldt andre uden at spsrge , om ikke ogsaa nogen kunde tilskrives ham <b>selv</b> . Ganske vist var der altfor meget af en Egenkærlighedens Aand overalt ; enhver tog først og fremmest Hensyn til sine egne Dnster og sin egen Villie og tcenkte blot derefter paa de andres . Men var dette ret ? Augustus havde af Naturen en oprigtig Charccktcer , stjsnt den var meget stolt , og han tilstod nu aabent for sig <b>selv</b> , ikke alene at det hele System paa Brocklehurst var feilagtigt , men ogsaa , at han <b>selv</b> bar sin Deel af Vildfarelsen . Dog her standsede han — den Sced var ikke bleven udsaaet i dans <b>Hjerte</b> , hvoraf vi lwre , at der her handledes om noget langt Vigtigere end endog huuslig Fred og Lykke — saa stor en Velsignelse disse Goder end ere — og at hvor der ikke sindes Lyst til nt behage og adlyde en Fader , paa Grund af at dette er den himmelske Faders Villie , der kan ikke være nogen <b>sand</b> Fred og <b>Kjærlighed</b> . Augustus var kommen i Vane med at ringeagte sine Sostre ; men Alice ringeagtede han ikte , og denne Morgen iscerdeleshed forekom det ham , som om hun stod meget langt over ham i Godhed . Han beslnttede at gjsre Forjog paa at overvinde sin Stolthed

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009090403022'target=_>Boisen, F.E., 1867, Nordiske gudesagn</a><h4>188</h4>maatte vare Sedet for Thor at komme til Hjalp , og at han kom og brugte sin Hammer til Asgaards Frelse , det har vi seet . Vi ter da vel ogsaa flntte , at det lykkes itte altid endnu for Hr ungner at forvandle ethvert Asgaard , som han astagger sit Veseg , til et lettehjem . Det er min Tro , at alle livlige Naturer faae et Beseg af ham ; ganske uden Paavirkning bliver hans Veseg neppe ; men han kan itte blot drives paa Deren, men endog stanes ihjel , nåar man itide faaer kaldt Thvr til ' Hjalp med hans Mjelner . leg har segt at fremstille Thor med Mjelner , som et Villede paa Mrlighed og Virksomhet » i <b>Sandheds</b> Tjeneste . Hvad vi dremte om som Bern , og hvad vi vare begeistrede for som Vorne , var vel i Grunden dette , og forsaavidt det var noget Ondt , da maa det jo gjerne dcc , stjendt det er itte det , Hr ungner vil drabe . Hvor nu <b>Oiet</b> er opladt for Ham , t » er itte blot arbeider i <b>Sandheds</b> Tjeneste , men er <b>selv</b> <b>Sandheden</b> , paa hvem Thor , der knuser Slangens Hoved og herer alle dem , som kalde paa ham , er et smukt Villede , — der veed man. at hvem det gjalder om at kalde paa ved et Hr ungner-Veseg , for baade at faae ham jaget paa Deren og staaet ihjel , det er denne almagtige Herre , der har et Ord , hvormed han traffer sittrere og kraftigere end Thor med sin Hammer . Men jeg veed , at det nytter itte at opfordre dem til at paakalde Christus , for hvem han endnu er ukzendt , og hvad jeg her skriver er itte blot strevet for Christne . men for alle danske Mand og <b>Qvinder</b> , der have <b>Hjerte</b> for Sandhed, og som grue for at blive en saadan Hr ungner , som i det Foregaaende er bestreven . Naar jeg derfor henviser dem til Thor og hans Hammer , da mener jeg dermed , at de dog vilde blive <b>Sandheden</b> troe . som er opgaaet for deres <b>Oine</b> , og itte flippe den for nogen vitterlig Legn . om den maastee kunde synes mere fordcelagtig , og jeg henviser dem dernast til Virksomhed i denne <b>Sandheds</b> Tjeneste , lige vigtig for Christne og Itte-Christne .


<h2>“Jul” [Christmas]</h2><h3Mor_, Kvinde_, Barn_, Gave_, Takknemlig_, Glede_, Hellig_</h3><h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092403020'target=_>Ahlfeld, Friedrich, 1865, Menneskelivet i Guds Ords Lys</a><h4>172</h4>mnn vente , indtil et fremmed Hjerte , forn har mere Kjerlighed end Moderhjertet , kommer og tager det til sig . Ve en faadcm Moder ! Hun betragter ikke sit Varn som nogen Guds <b>Gave</b> , men som en Byrde , der er hende paalagt mod hendes Villie , som en Anklager mod hendes Synd og Skyld . Huu veed intet af det Ord : „ Men nåar hun har fodt Barnet , kommer hun ikte mere den Trengsel ihu , nf <b>Glede</b> over nt et Menneske er fodt til Verden . " Trengselen lever med Barnet , ng derfor vil hun vere fri for det . — Dog , vi ville ikte tale mere om disse <b>Modre</b> uden Moderhjerte . Men ner til dem grendse de <b>Modre</b> i de store Byer , som efter Fonselen sette sit <b>Barn</b> i en af de Pleiestiftelfer for fattige Dicborn , forn gjerne der findes . Naar faadnnne <b>Born</b> begynde nt face lidt Forstand , moder hellerikke deres Oie nogen Moder , men en fremmed <b>Kvinde</b> , forn gjor Smnnborns Opfodelse — Opdragelse kunne vi ikke kalde det — til et Hacmdverk , et Levebrod . Moderen hnr frasagt sig Moicn med sit <b>Barn</b> , men dermed ogsaa Gleden deraf . Ikke stort bedre er det i de fleste Tilfelder , nåar <b>Born</b> , hvad man kalder , udfettes til Opfostring . Om end dette Skridt ofte knn vere en forgelig Nodvendighed , faa er denne dog fordetmeste en Frugt af Synden . Snndnnne <b>Modre</b> stulde kim ret betenke , hvorledes Synden forgifter nl <b>Glede</b> , hvorledes den losner felv de inderligste Forholde , saaledes som Barnets til Moderen . Sjelden , sjelden betragter Barnet senere Moderen som sin rette Moder . Hun har jo hellerikke selv betragtet Barnet rigtigt som sit <b>Barn</b> , har ikle med snur Moie og vaagne Netter kjobt sig en Plads i Barnets Hjerte . Gud havde givet hende det som en <b>Gave</b> , og hun har ikke villet have det som saadan . — Men selv bortseet fra disse Afveie , staaer ofte Sorgen i <b>Gledens</b> Sted ved det stakkets Barns Vugge . Istedetfor Tnknemmeligheds- og Gledesstraaler feer Barnet Knmmerens Taarer i Foreldrenes Oine . Istedetfor den forste Vuggesang horer det Sukke . Ofte teuke fattige Foreldre : „ Varneflokten bliver for stor . Hvem stal sode den ? Vi kunde med Nod og Neppe staffe Brod til dem , vi havde , og nu stjeuker Gud os eet til ! " Mangen Fader har allerede knurret under siv Hustrues Svangerstab og siden seet suurt til sit yngste Barns Vugge . Og dog er dette ilke Andet end Mangel paa Tro , Mangel paa Mod . Dn er ikke Barnets overste Fader . Dets Skjebne styres af Ham , fra hvem alt Fadernavn er i Himlene og pnn Jorden . Hau hnr cmtaget det til sit <b>Barn</b> i den <b>hellige</b> Daab ; Han har ladet det dobe til sit Navn ; Han har ikte fagt : „ Flokk.n bliver for stor ; jeg vil ikke mere lade noget <b>Barn</b> dobes til mit Navn , min Naade og Omsorg . " Nn

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009091503033'target=_>Ahlfeld, Friedrich, 1866, Menneskelivet</a><h4>590</h4>har Du antaget os som saadanne og er bleven os en kjcerlig , trofast Fader . Lad alle Stedforcrldre speile sig i denne naaderige Faderkjccrlighcd; lad der fra dette Vceld stromme faderlige og moderlige Folelser i deres Hjerter . Men Du tilskynde ogsaa alle <b>Born</b> , hvem Du skjcrnker en anden Fader eller Moder , til at modtage dem som dine dyrebare <b>Gaver</b> , til at komme dem imode med hjertelig Tillid , ikke at vanskeliggjøre dem deres Stilling , men at takke og bede flittigt for dem og trcede i det inderligste og hjerteligste Samfund med dem . Du var ikke vor Fader , og dog have vi ved din Kjcerlighed lcert at elske Dig og slutte os hjerteligt til Dig som kjerre <b>Born</b> til deres kjcere Fader . Lad hine stakkels <b>Born</b> see hen til denne vidunderlige Erfaring, og lad dem fatte en ovrigtig Kjcerlighed til dem , forn deres himmelske Fader giver dem til Fcedre eller <b>Modre</b> . Du bygge Huset af Naade i den <b>Hellig</b> Aand . Amen . Kjcere Lcesere ! Naar Doden stiller en Mand og en <b>Hustru</b> ad , som have elsket hinanden , saa er den Tanke som oftest levende hos den tilbageblevne Mgtefcelle :  leg vil ikke gifte mig igjen . Der gives dog kun een fsrste Kjcerlighed , kun eet Foraar i Livet . Jeg vil ikke give mine <b>Born</b> en Stedmoder eller Stedfader ; jeg vil vcere baade Fader og Moder for dem og ovdrage dem alene . " Og Mange er det blevet givet af Herren at kunne udfore dette Forscet . Vi ville ikke tale om de Enker , der have afviist enhver Tanke om en ny Forbindelse, der have udfyldt deres Liv med at tjene Herren , med Mindet om den bortgangne Mand og med Kjcerligheden og Omsorgen for deres <b>Born</b> . Vi have ogsaa kjendt Mcend , som , efterat vcere blevne Enkemcend i en nng Alder , aldrig kunde beslutte sig til at indgaae et andet 3 Egteskab og hellere gave Afkald vaa alle selskabelige <b>Glceder</b> , end at de skulde have bragt deres <b>Born</b> en ny Moder i Huset . Vi have kjeudt saadanne Mcend , som efterat have Paataget sig det dobbelte Kald at vcere baade Fader og Moder for deres <b>Born</b> , udforte Alt , hvad der var muligt at gjore , som bedende kncelede med Bornene om Aftenen ved Sengen , og forn i Hnusvcesenet sorgede for alle deres Fornodenheder . Dog dette er ikke givet Alle . Mangen Fader maa som en Folge af sin Charakieer og sit Kald soge en ny Moder for stue <b>Born</b> , og mangen Moder takker Gud , naar en vakker Mand tilbyder hende sin Haand , for at vcere hende og hendes <b>Born</b> en Hjcelp og Ststte i Fremtiden . — - Det er vel sandt , at blandt alle de Christne , som omtales i det nye Testamente , ncevnes der Ingen , der har indgaaet et andet AFgteskab . Men den <b>hellige</b> Bog giver heller ingen Familiehistorie eller Skildring af Sccderne , men Frelsens Historie . Paa den anden Side findes der intetsteds noget Forbud af Herren og Apostlene mod at indgaae et nyt 3 Egteskab . Naar Herren siger :  Den , som gjorde dem af Begyndelsen , gjorde dem Mand og <b>Kvinde</b> " , og  de To stulle vcere til eet Kjod " ( Matth . 19 ) , saa forbyder han derved ikke det andet 3 Egteskab , men Fleertoneriet , der var blevet Skit ikke alene hos Hedningerne , men ogsaa hos den gamle Pagts Folk , og havde bragt megen Ulykke og Forstyrrelse i


<h2>“Kvindens funksjon i den offentlige sfære” [Woman’s function in the public sphere]</h2><h3Kvinde_, Almene_, Sædelighed_, Land_, De uprivilegerte _</h3><h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008082512001'target=_>Luther, Martin, 1863, Dr. Martin Luthers Udførlige Udlæggelse af første Mosebog</a><h4>191</h4>holder sig i Gudsfrygt og beder , end at man stnlde cengste og plage sig med Frygt for de tilkommende Ting . Men derom en anden Gang . 127. Saa er det nu min Mening , at man ilke saa sikkert kan grunde de astrologiske Forudsigelser paa denne Text ; thi det er Tegn , som Fornuften iagttager og samler . Men den ncermeste og egentlige Mening er denne , at vi forståa , at Moses taler om de Tegn , som Gud viser os , foråt Mennestene i Almindelighed derved skulle enten formanes eller forfcerdes . Og dette stal vone nok om den fjerde Dag . 128. Men her antydes for os Sjcelens Udodelighed , eftersom ingen Skabmng uden Mennestet alene kan forståa Himlens Lob eller maale og begribe Himmellegemerne . Et Svin , en Ko eller en Hund kan Ne gjore sig noget Begreb eller nogen Tanke om det Vand , som de drikke ; men Mennesket har Begreb og Skjon paa Himlen og alle Himmellegemer . 129. Derfor skinner her en Gnist as det evige Liv , at Mennestet af Naturen maa ove sig i saadanne Tings Betragtning og Erkjendelse . Og denne Omsorg viser , at Mennestene Me ere skaute til , ultid at leve paa Verdens laveste Sted , men at de tilsidst skulle jaa Himlen i Eie , hvilken de i dette Liv betragte med Forundring og ove sig i Betragtning og Studermg paa himmelske Ting . 139. Og hvis Ne det var saa , hvortil havde det da tjent , eller hvorfor havde det ogsaa vceret fornodent , at Gud saa rigelig havde ladet Mennestet vide saadanne Ting , hvem dets eget Legemes Skikkelse og Retning give Vidnesbyrd om , at det horer til de himmelske Ting ; hvorvel dets Begyndelse er same svag , <b>fattig</b> og elendig . Thi det forste Menneske blev gjort af en lordklump ; derefter begyndte den menneskelige Slcegt at avles og formeres ved Mandens og <b>Kvindens</b> <b>Sced</b> . Deraf bliver efterhvert Fosteret i Moderlivet uddannet med alle Lemmer og voxer , indtil det ved Fodselen kommer til Dagen og dette Himlens Lys . Derncest begynder det forst at fole , arbeide og rore sig . 131. Men nåar ester alt dette Legemet er kommer til sin Kraft og Styrke , og i et sundt Legeme Fornuften og Forstanden er i Virksomhed , da viser sig forst det forstandige Liv , som Ne er i andre jordiske Skabninger, saaledes at Mennesket ved de mathematiste Videnstabers Uedvirken og Hjcelp , om hvilke Ingen kan sige Andet , end at Gud har aabenbaret og vist dem , fra Jorden styver med sit Sind og sin Tanke til det Hoie , lader de jordiske Ting fare og udforsker og betragter de himmelske . Dette gjore ilke Kjor , Svin eller andre ufornuftige Dyr ; det gjor Mennestet alene . Derfor er Mennesket en Skabmng , som er stabt til endelig at forlade , Jorden og i Himlen leve i en evig Tilvcerelse . Thi deraf kommer det , at han Ne alene kan tale og domme ( hvilket henhorer til Dialektik og Rhetorik ) , men ogfaa kan lcere den hele Lcere om Himlens Lob , Stjernernes Bevcegelfe og hvad den menneskelige Fornuft mere kan begribe af disse hoie og hemmelige Ting . 132. Derfor aabenbares nu fra denne tredie Dag af vor Wre , at nemlig Gud nu tcenker og foretager sig at stabe en saadan Skabmng , som kan erkjende og forståa Bevcegelsen af disse Himmellegemer , der ogsaa stabes paa den fjerde Dag , og fom kan have Glcede af en saadan Erkjendelse , hvortil ogsaa dens Natur er scerdeles tilboielig , og som passer sig for den . Men alt dette skal bevcege og opvcekke os til Tcitsigelse , at vi som Vorgere henhore til dette <b>Fcedreland</b> , som vi nu se paa med Forundring og forståa , men endnu kun som Fremmede og Landstygtige ;

<h4>3865</h4>forjcettet af Gud ; og dog besad han Me en Fodbred i hele Landet , men vankede omkring med Sine , er snart paa dette , snart paa hint Sted , og har slet ingen bestemt Bolig . Og ester hans Exempel droge ogsaa hans Born og Borneborn saaledes omkring i Elendigheden , og haude intet bestemt Sted , hvor de kunde bo . Men hvortil tjener nu det ? Svar : det tjener til , at vi skulle vide , at denne hellige Patriark ventede paa et andet Rige og et andet <b>Land</b> og Arvelod , end hvad der er her paa Jorden og i dette Liv ; derfor forblev han en Fremmed i sit eget <b>Land</b> , som var ham forjcettet as Gud , nemlig foråt efterlade Pharisceerne den hellige Skrifts sikkre Underretning om Guds Rige og om Forjcettelsens fornemste Hovedstykke , nemlig at Kanaans <b>Land</b> ikke var det fornemste Stukke as den ; thi ellers havde ikke Abraham faaet Forjcettelsen , siden han jo levede som en Fremmed i det forjcettede <b>Land</b> tilligemed sine Borneborn . Derfor maa ogsaa vi leve ligesom Fremmede i dette Liv , iudtil vi naa det rette <b>Fcedreland</b> og erholde et bedre Liv , soin er evigt . Men de , som levede ester Fcedreues Tid , forglemte den evige Forjcettelse , saaledes som Pharisceernes Historie lcerer , og gabede kun ester Besiddelsen as Kanaans <b>Land</b> og ansaa derfor Christi Rige for et legemligt Rige . 9. Men Propheterne og de andre Hellige have flittigere betragtet denne Abrahams Pilgrimsstcmd , og paamindet om , at man maa vente paa et andet <b>Land</b> og et andet <b>Fcedreland</b> , end dette legemlige og forgjcengelige . Thi ellers var dette legemlige <b>Land</b> ogsaa bleven givet Patriarken Abraham , hvem det var forjcettet ; men i hans hele Livstid blev der ikke engang givet ham en Fodbred deri ; ja , han maatte for egne Penge kjobe det Sted , hvor han begrov sin Sara . Det Have Propheterne seet ; men vore pavelige blinde Muldvarper kunne og ville ikke se det , men sige : Abraham havde en <b>Hustru</b> , derfor var han en simpel almindelig Lcegmand ; derfor er der heller intet Aandeligt i hans Historie , Alt er ganske almindeligt o . s . v. 10. Men Brevet til Hebrceerne dommer anderledes derom . Thi det iforer disse almindelige Ting Aanden og siger Kap . 11 , 8. 9. , at disse almindelige Ting stede i Troen ; at han i Troen tog ' en <b>Hustru</b> , at han i Troen var Fremmed her og der ; det er , at han altid holdt sig til den guddommelige Forjcettelse eller Ordet , og ventede paa en anden Stad eller Bolig , end denne jordiske ; denne brugte han kun paa samme Maade , som han brugte Mad og Drikke . Derfor stulle vi ogsaa lcere at bruge dette Liv saaledes , nemlig som et almindeligt Herberge , hvor vi kun opholde os een Nat . 11. Og hvis du paa denne Maade forstaaer Abrahams Pilgrimsstand, at han var en Fremmed i Landet , da vil du ikke sige , at dette var noget ganske Almindeligt ; thi det er en Troens Gjerning , der i Abraham var saare brcendende og stcerk . Men de Ugudelige se ikke dette , som strevet staaer Es . 26 , 10. : den Ugudelige ser Me til Herrens Hsihed . Thi de Gjerninger , som have sin Oprindelse af Ordet og ste i Troen , ere fuldkomue for Guds Vine , Verden maa derom domme fom den vil , selv om du tun er en <b>fattig</b> Faarehyrde eller Amme . Men , som jeg har sagt , de blinde Muldvarper se kun hen til de Ting , som have et scereget Skin . Om du paa et eller andet <b>offentlig</b> Sted vilde udstille en Abekat , som dog er et stjcendigt , unyttigt Dyr , da vilde du strår finde Folk , som lob til Stedet og sorundrede sig over Abekatten som et saare stjont Dyr . Saaledes bcerer Verden sig ad . Hvad der er

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016101948051'target=_>Nicolaysen, N., 1858, Norske Stiftelser</a><h4>3198</h4>i Christiania lorgen Holm og <b>Hustru</b> Maren Ssrensen ved Gavebrev af Oktbr . 1834 ( II . 533 — 536 ) en Kapital af 1000 Spd . , som skulde blive staaende mod Iste Prioritet i deres Gaard Matr . No . 10 i Storgaden og hvis aarlige Renter efter Vegges Død skulde tilfalde deres 2 de Born Ingeborg Lagertha og Randine Andrea , saalænge de trcenge til saadan Understottelse , men senere anvendes til Opdragelse af norske indfodte , <b>fattige</b> , sedelige og velartede Vorn , saaledes at Stifternes trcrngende Afkom , udeu Hensyn til Fode- eller Opholdssted , og Born efter bortdsde Forceldre , der har været hcedcrlig bekjendte for virksomt Fcedrenelandssind og bosatte i Christiania , ere fortrinsberettigede . Bornene skulle anbringes i en <b>offentlig</b> Opdragelsesindretning , „ hvor Kjcrrlighed til vort kjære <b>Fcedreneland</b> og Vekjendtstab med dets Historie og Forfatning ikke lades af Sigte ved Opdragelsen « ( i Mangel af nogen anden passende Indretning , kan vcrlges Eugenias Stiftelse ) , og naar et Barn er konfirmeret , cmsees dets Opdragelse fuldendt , hvorefter det for Legatets Regning førsynes med anstcrndigt UdM af Klæder og andre deslige Fornodenheder .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015052748022'target=_>, 1854, Almennyttigt dansk Konversations-Lexikon over alt det Videværdigste i Naturen, Kunsten og Videnskaben, fra den ældste til den nyeste Tid, i alfabetisk Orden, efter Brochhaus's Tydske Original, nyeste Udgave</a><h4>430</h4>Spanien , Osterrig , og i Orienten , navnlig i China , hvor den endnu har en Missionsplads . LazeV eller Laser kaldes Beboerne i det Tyrkiske <b>Land</b> La si stan i Lilleasien , den sydostlige Kyststrakning ved Det sorte Hav , og som mod , O . grandser til Georgien , der ved en streng Russisk Militcercordon stilles herfra . Lazcrne forraadcr deres Slagtskab med de «armest Kaukasus boende Folkeslag mindre ved deres Legems- og Ansigtsdannclse end ved deres Sprog , en Mundart af den Iberiske Sprogstamme, og ved deres raa <b>Scedcr</b> , navnlig ogsaa ved den Strenghed , hvormed de udover Blodhavnen. Paa Grund af deres voldsomme og rovcriste Charakteer staaer de i ilde Ny hos Tyrkerne og Georgierne . Ofte foretager de Rovertog ind paa Russisk Gebet , og af Had til Russerne letter de Grandscsoldatrrnes Desertion. Allerede heraf lader Ruslands Begierlighrd efter at komme i Besiddelse af Distriktet Batum sig forklare . Den vigtigste Handelsplads er Vatum eller Batum i . ( 2 ) Lazise , en Flekke ved Garda-Socn i Lombardiet-Venedig. er befastct , har en bekvem Havn og godt Fiskeri . 3000 Indb . ( 2 ) LNzo , se Lasso . Lazursten , se La sursten . Lazzar « ni eller L azz a roner ( It . ) < pi . , husvilde , <b>fattige</b> Folk , SioUere , Lastdragere , Tiggere , Gadepobel i Neapel og Sicilien , ogsaa kaldte Banchicri ( udt . Bankjeri ) , fordi de som oftest ikke har andet Leie end en Bank under et Skur . L a zz a r one r er en , som man mener , af den syge Lazarus laant Benavnelsc paa en i sin Art eneste Klasse af Neapels Beboere. Uden egentlig Stand , bestemt Besiaftigelse, fast Bolig eller sikkert Underhold , lever de fordetmeste Dag og Nat hele Aaret igienncm paa Gaderne og de <b>offentlige</b> Pladser , hvor de uden anstrengende Arbeide fortiener deres faa Fornodenheder som Bude , Lastdragere , Dagleiere o . s . v. Ligesaa godmodige som cyniske i sadelig og legemlig Henseende , ligesaa skiodcslose som dovne , viser de dog i hoieste Grad den Syditalienske Charakteers stolende Livlighed og er kun altfor tilboielige til Uroligheder , ligesom de da ogsaa i alle Revolutioner og Folke « bevagelscr i Neapel har spillet en stor Rolle , snart for det ene , snart for det andet Parti . I den nyere Tid har de nastcn altid sluttet sig til det conservative Princip . Ligesom Pobrlcn ' i Paris i Middelalderen valger de aarlig en egen Hovding , Capo Lazzaro , som Regieringen formelig anerkiender , da den ved ham lettest formaner at beherske denne Neapels Gadepobel, der taller henved 60,000 Mennesker . Da Neapels Beliggenhed frembyder Alt , hvad der overhovedet gior en saadan Leveznaade mulig ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015010648112'target=_>, 1853, Almennyttigt dansk Konversations-Lexikon over alt det Videværdigste i Naturen, Kunsten og Videnskaben, fra den ældste til den nyeste Tid, i alfabetisk Orden, efter Brochhaus's Tydske Original, nyeste Udgave</a><h4>6677</h4>Kumisbrcendevin , af Kalmukkerne kaldet Wina eller Racky , som ved Destillation vindes af Kumissen . ( 1 ) KttMykerne , et af 6 — 7000 Familier bestaaende Tatarisk Folk i den sstlige Del af Kaukasuslandet , ved Terek og det Kaspiske Hav , beboer et biergigt , men frugtbart <b>Land</b> og lagger Vind paa Kvcegavl , Fiskeri , Agerdyrkning , nogen Vin- og Bomuldsavl . Mandene arbeider i Solv og Jern , <b>Kvinderne</b> vave smukke Toicr . De boer i ordentlige Byer , bekiender sig til den Muhamedanske Religion og staaer under Khaner , af hvilke Khanerne i Aksai ( Arai ) og Endery er de mcrgtigste . Kumykcrne crkiender vel Ruslands Overhcrrcdomme , men er dristige Rovere og driver Slavehandel . Forsiielligc fra dem er Kasi-Kumykerne , hvilke ogsaa er Muhamedanere og et krigersk Roverfolk, men horer til Lcsghiernes Stamme og boer lcrngerc mod V . ; deres vigtigste Byer er Kunyk og Kasanisch , begge velbefolkede og Seedet for foisiicllige Stammcfyrster . ( 2 ) KnnlTra . en By i Babylonien , ved den ostlige Bred af Euphrat , omtrent 10 M . fra Babylon , blev markelig ved Slaget 401 f . Chr . mellem Artaxcrxes Mmenon og hans Broder , den yngre Cyrus , som faldt i Kampen . ( 2 ) Kunra , en 1815 anlagt Havn og Flette i det Russiske Gouvernement Reval , ved den Finske Bugt , har stcrrk Udforsel af Tommer , Korn og Braudevin . ( 2 ) Kundurilltis ( Lazaros ) , en af sit <b>Fcedrelands</b> Befrielse hoilig forticnt Graker , blev fsdt 1768 paa Den Hydra , hvor han ligesom sin Broder Georg K . levede som en af de rigeste og anseteste Skibsrhedcre . Da den Grerste Uafhangighedskamp begyndte 1821 , bragte Vrodrene Fædrelandets Befrielse uhyre Pengcoffre ( der siges Mill . Francs ) og otte Skibe , saa at de som Folge heraf selv blev <b>fattige</b> . Uagtet Lazaros hverken deltog umiddelbart i Befrielseskampen eller i Landets Bestyrelse , gavnede han dog overordentlig Fædrelandets Sag ved sine kloge Raad , sin Udholdenhed og sin store Indflydelse paa sine Landsmand . Han var Prasident for sin Fodeoes Senat , og under Kampen sictc ikke lettelig Noget uden hans Minde . Han bevarede ogsaa sin crdle Charaktcer, da senere under Kapodistrias ' s Prasidentsiab, nnder Regentskabet og selv i hans sidste Levetid Utak og Bagvaskelse sogtc at nedsatte hans Fortienester . Efter at have giort mange frugteslose Stridt for at skaffe de ved Offrene for deu <b>almene</b> Cag daarc t medtagne L > > - Hyrra , Spez ; ia og Ipsara Hielp af Etat , » , dode han 1852 paa sin Fcedrco . Negieringen oerede nu ham og hans Fortienester , id » t deu anordnede , at Embedsmandene og si ^ ! ^

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112301019'target=_>, 1859, Norsk lovsamling for Aarene 1667 til 1858</a><h4>955</h4>til disse <b>offentlige</b> Gjordemoder - Indretninger . 18 ) Saa stal og enhver Kjsb » stad og ethvert £ anbbiflrict forsyne den ansettende © jorbemober med de Redstaber , som ansees fornodnc til hendes Bestilling . Disse anffaffeé førfte Gang paa Kjsbstadens etter Districtets Bekostning , og overleveres Gjordemoderen ved Syn , samt tilbakeleveres paa lige Maade . 20 ) Naar Veiens Lengde etter Omstcendighederne krever , at Gjordemoderen maa betjene sig af Vogn , da bor de , som begjcere hendes Hjcelv , enten selv lade hende afhente og hjemfsre , eller og godtgjere hende de Omkostninger, hun paa nodvendig Befordring maa anvende . 21 ) Ligeledcs al Gjordemsdrene i Landdistricterne , naar de kaldes til at betjene <b>Fattige</b> , eller Huusmends og Indersters Hustruer , der ikke selv have Heste og Vogn , nyde fri Befordring frem og tilbage . Denne Befordring paaligger bet Districtets Gaardmend at afgive ester Omgang , og paa Tilsigelse af Sognefogden eller Oldermanden ' ) . 22 ) Derimod flat samtlige Gjordemsdre , som efter det Foregaaende ere ansatte med fast Lsn , uden Betaling betjene alle de <b>Fruentimmer</b> , som med Attest fra rette Vedkommende godt- » gjøre , at de nyde Underststtelse af Fattigvesenet . 23 ) Under disse Vilkaar stal t ethvert Gjordemoder-District snarest muNgt anscettes en oplert Gjordemoder , som bringes i Forslag af Stedets Dvrighed efter Overlceg meb Districtets Lcege , og beflikkes af Oversvrigbeden . 24 ) Naar der iet District opholde sig flere eraminerede Gjordemsdre , og Vacance indtrceffer blandt de Gjordemsdre , ber have Lsn , da bør Dvrigbeden , » eb at befcette den ledige Plads , fornemmelig tåge Hensyn til ben ulsnnede Gjorbemoder sammesteds , ber bar udmerket sig ved Flid og Duelighed ; uden at ben tcengere eller kortere Tid , i hvilken hun har veret ® jorbemober , stal komme i Betragtning. 25 ) I Almindelighed maa Ingen bestikkes til Gjordemoder under ovennevnte Vilkaar , medmindre hun er forsynet meb Testimonium om sin Dueligbed fra den Kgl . Gjordemoder-Commission ; faa bør hun beéuben have forbeetag * tigt Vidncsbyrd om <b>Sedelighed</b> fra hendes Sjelesorger . 26 ) Skulde der ved denne Indretnings Iverksettelse ikke sindes et tilstrekkeligt Antal af ssige ordentlig oplerte Gjordemsdre . ba vil Kongen , paa bet at intet Gjordemoder-District stal savne Fsdselsbjelpcre, have Oversviighederne bemyndiget til t ssige Tilfelde at give saadanne <b>Fruentimmere</b> Tilladelse til at practisere som Gjordemsdre , der enten ere blevne oplerte af © ttftø- og <b>Land</b> - Physicis , og af disse have erholdt Vidnesbyid om deres T ^ upTtg ^ pb i psspv og , efter Dvrighedens Foranstaltning , af Stifts- eller Land-Physico , i Overveielse af Distriktslegen eller anbcn lcegekyndig Mand , ere blevne prsvede og befunbne og have den Kundstab og gcerbfgtjeb , at det , uden at befrygte nogen Fare , kan tillades bem at betjene Sarfelquinber . 27 ) Disse af Stifts - eller Landphysicis saaledes oplerte etter prø » ebe <b>Fruentimmere</b> , hvilke af Mangel paa ordentlig eraminerede Gjordemsdre ansettes i Districterne . stal ikke tillegges den for Districts-Gjordemsdre i de foregaaende §§ bestemte £ øn etter de øortge faste Emolumenter , mebminbre de , af Mangel paa duelige Subjecter i selve Districtet , kaldes bertit fra et andet Di- » strict , og altsaa maa stytte fra deres Hjemstavn . Derimot » ssal de vere berettigede til at nyde ben i §§ 19 , 20 og 21 bestemte Betaling og frt Befordring , samt , naar de betjene de i § 22 nevnte <b>Fattige</b> , da tillige Betaling af vedkommende Fattigvesen ; ligesom det ogfaa ssal vere bem uformeent at vedblive deres Praxis , om endog en ordentlig examineret Gjordemoder bliver ansat i Districtet , hvilket bsr stee . saasnart Omstendighederne tillade det . 29 ) Naar der til et Gjordemoder-District ikke kan erholdes nogen ordentlig oplert Gjordemoder , ba stal Stedets Dvrighed derom gjøre Indberetning til Oversvrigheden , samt bringe eet eller flere Subjecter , helst af Di » strictets egne Beboere , i Forslag til at opleres t © jorbemober » ibenff aben , for ftbeit at ansettes som Gjordemoder i Districtet . 30 ) Det <b>Fruentimmer</b> , ber bringes i Forslag til Oplerelse som Gjordemoder , maa ikke vere under 20 Aar og iffe over 30 Nar gammel ; hun bør have Beviis fra sin Sjelesorger om benbeé © cebetigbeb , og Attest fra en Lege om hendes Physiste Beqvemhed ; hvorved tittige maa iagttages , at hun ikke bar nogen Legemsbrek eller Mangel , som kunde opvekke Skrek eller Afsiye hos den Fsdende ; ogsaa stal , under lige Beqvemhed til at bringes i Forslag , den nydeFortrin , som er gift og selv har føbt Søm ; tigefom og den , " ber kan skrive , bør foretrekkes Andre , der ikke forene denne Ferdighed med de svrige udfordrende Egenstaber. 31 ) Dersom der iblandt de <b>Fruentimmere</b> . som saaledes bringes i Forslag , stulve findes Nogen , der besad alle fornodne Egenstaber , men ikke havde Evne til selv at bekoste sin Oplerelse . da stal Oversvrigheden . naar den sinder saadant efter Omstendighedcrne nsdvendigt , vere bemyndiget til at foranstalte bende paa bet Almindeliges Bekostning underviist ved Lereanstalten for © jorbemøbre . 32 ) Omkostningerne paa en faaban gærltngg Dp ^ otb og UnberoiiSntng i Stiftelsen , samt paa hendes

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014082608023'target=_>James, John Angell, 1872, unge Kvinde</a><h4>89</h4>til yderlighed overdreven Vurdering af <b>Kvindens</b> Rettigheder og Forrettigheder end en bevist Indrvmmclse af dem , betragtede og opfattede af Fornuftens Lys og i Overensstemmelse med Guds Bud og Forskrifter ; det kan ogsaa vcerc et Sporgsmaal , om ikke mangen tapper Ridder , naar Indbildningens Glans var blegnet og Lidenskabens Ild udslukket , kunde knuse det Hjerte , han havde veeret saa ivrig for at mnde , og dette just fordi Kristendommens Aand var ham fremmed . Tet er Kristendommens Mre , at den , istedet for at virke paa Indbildningen , Sanserne og Lidenskaberne, meddeler en Aand , som tager Bolig i Sjoelen og behersker Samvittigheden , og iftedet for at forfsre Manden til taabelig Beundring af <b>Kvindens</b> Inde ved <b>offentlige</b> Forlystelsers pirrende Optrm , leerer og leder ham til fsrst og sremst at betragte hende og nyde hendes Omgang i Familielivets übemcerkede Fortrolighed og i Hjemmets hellige Kreds , Kristendommen tillader ikke dette taabelige Afguderi , som Riddervcesenct befordrede , og fom just paa Grund af fin yderlighed maatte esterfolges af Ligegyldighed eller Foragt . Kristendommen gjor af <b>Kvinden</b> hverken en Guddom eller en Slave den gjor hende til Mandens Ven og Ledfagerinde gjennem Livet og leorer ham at betragte hende i dette hcedrende og elskelige Lys . Kunne vi ikke i alt dette se et skjsnt Bevis for vor hellige Religions umaadelige Fortrin ? I enhver Henseende , om vi betragte dens Virkninger for denne eller den tilkommende Verden , dens Forhold til <b>Samfundet</b> etter til Gud , dens hoie Lcerdomme eller dens rene <b>Moral</b> , se vi her et Billede af uefterlignclig Skjpnhed , mcegtigt til at indtage ethvert Hjerte , fom ikke er blevet forstenet ved falsk Filosofi, aabenbar Vantro eller grov llscedelighed . Men i ingen Henseende er det skjonncre end med Hensyn til <b>Kvinden</b> . Med hvilken Ligelighed afveicr den tkke alt mellem begge Kjon ! Med hvilken Mrldhed holder den tkke sit Skjold over det svagere Kar ! Med hvilken Visdom opretholder den ikke og forsvarer deres Stilling og Rettigheder, hvis Indflydelse er faa vigiig for <b>Samfundet</b> , og begrcondser alligevel deres Fordringer faaledcs , at de ikke

<h4>173</h4>Kristendommens , Merlandenes eller Vesterlandenes , Fortidens eller Nutidens , og vi skulle finde , at intet saa voesentlig adskiller dem som <b>Kvindens</b> Tilstand . Enhver <b>Kvinde</b> , hun vcere nu gift eller ugift , rig eller <b>fattig</b> , har dog en Kreds , indenfor hvilken hun udover en vis Magt , enten til Skade eller Gavn . Enhver <b>Kvinde</b> vil nødvendigvis ved sin Daarstab eller sin Klogskab , sin Dyd eller sine Laster forege sit Folks Vcerdighed eller paaskynde dets Forfald . Saalcengc <b>kvindelig</b> Dyd blomstrer , beskyttet af det ene og agtet af det andet KjM , kan et Folk ikke synke altfor dybt . Til en vis Grad er det <b>Kvinden</b> , som opretholder sit Folks Lykke . Hendes Dyd staar , hvis den er fast og ufordærvet, som en beskyttende Vagt over Folket . Lov , Retfærdighed og Frihed , Kunst og Videnskab , alt bidrager naturligvis til et Folks Velgaaende , men hvis <b>Kvindens</b> almindelige <b>Sædelighed</b> staar saa er alt andet sorgjceves , medens derimod , hvis <b>kvindelig</b> Dyd og Dannelse er almindelig, Civilisationens Stro ' m stiger til sin Drste og bliver frugtbringende for den mest udstrakte Overflade . Et <b>Samfund</b> , hvori <b>Kvinden</b> opfylder sit Kald , falder ikke lettelig ; thi den Magt , hendes cedlc Hjerte udMer paa andre Hjerter , vil lMe det af Elendigheden og gjengive det Glcede og Lykke . Se der , faavcl udenfor fom indenfor det ægteskabelige Liv hur <b>Kvinden</b> en Mission , og det ei en übetydelig . Hendes Virkekreds er Samsuudet , hendes Maal er dets Velbefindende , hendes BelMning dets Taknemlighed og Erkendtlighed .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111903047'target=_>, 1838, Samling af Norge vedkommende Kgl. Rescripter, Resolutioner og Collegialbreve m.v. for Tidsrummet fra 1660-1813</a><h4>4695</h4>? 39. Alle Born i Staden , som ikke have Foreldre eller Formyndere, der kunne , eller andre , der ville berale for deres Skulegang , bor gives fri Underviisning , og Fattigvesenets Direction bur drage Omsorg for og medvirke til , at de nytte samme . F 40. Saasnart Directionen for Fattigvesenet er traadt sammen , setter den sig i Correspondence med Directiunen over det Kongsbergffe Skolefond , samt vedkommende Kongelige Departements , for at faae saa mange af de Skoler Fondet hidtil har vedligeholdt, som behoves til de <b>fattige</b> Vorns Underviisning , sig ooerdragne tilligemed de fornodne Indtcrgler t , l at bestride Udgivterne ved Underviisningen, samt til at bevirke andre Forandringer ved Skolevesenet , som maatte ansees nodvendige eller gaunlige for Fattigvesenets Oiemeed . F 41. Skolegangens Oiemced bor vere : at Bomene ved deres Sjels ug Legems Krefters rigtige Udvlkling og Anvendelse kunne dannes til oplyste , gode , arbeidsomme Mennester og Burgere ; at de til den Ende tldligen hindres fra Lediggang og Laster , og vennes til <b>Sedelighet</b> ) , Orden og Ncenligbed ; at de anfores til saadan Kundstab , sum er noduendig og gavnlig fur Mennesket og Borgeren i Almindelighed , og at oe undervises og oves i saadant Arbeide , hvorved de i Fremtiden , enten i Mangel afandet Neringsar » beide , kunde fortjene deres Underholdning , eller nyttigen beskjeftige sig i ledige Timer . F 42. Underviisningen i Skolerne stal altsaa vere deelt imellem V.denstab , Haandarbeide og nyttige Legems - Ovelser . F 43. Ved Videnstab forstaaes ikke her det , sum henhurer til blut Lerdum , eller blot speculativ Kundstab i egentlig Forstand , men det , som for den menige Almue er furnodent , fatteligt ug brugbarr , saasum : Lesning , Skrivning , Regning , Religion og Mural , almindelig Kundstab om Naiuren , dens Indremlnger og Virkninger , om borgerlige Indretninger og Pligter , om de vigtigste Sundhedsreglcr , om <b>Fedrelandets</b> Forfatning og Beliggenhet ) vg deslige , alt efter korte og simple Læreboger , som dertil kunne velges , og med Hensyn til den Forstjel , sum bor gjures i Underviisning for Drenge vg <b>Piger</b> . Ogsaa sorges sor , at Bomene anfores til rigtig ug harmonist Sang , iser Koraler og gode Folkesange- F 44. Haandardeider bor vere af det Slags , som i det daglige Liv kan sur Almuens Fult vere al » mindeligt og gavnligt , og sum for Bomene efter Furstjellighed af Kjon , Alder og Krefter findes passende , samt hvortil i Skulebygninaeme kan vere Leilighed og Rum , saasom : at pille Vomuld , at karde Uld og Bom » uld , at plukke og spinde Uld , Bomuld , Hor , Hamp og Blaar , at tvinde , haspe , vinde , spole , at binde og strikke , at kniple , at lappe paa Kleder , at stoppe Strumper , at knytte Fistegarn , at veve Bendler og uldne Baand , at gjore Hegler og Fistekroge , stette Kurve og Hatte , at forfcrrdige allehaande Legetoi og andet Numberger-Kram , og andet mere ; og for <b>Pigerne</b> tillige at tildanne og spe Linned og Kleder , m . v. Ved Arbeidets afvexlende Afdeling sees blandt andet ogsaa derpaa , at Legemskrefterne styrkes tilligemed oves . F 45. Til at give Underviisning i Haandarbeide ansettes ved hver af Fattigvesenets Skoler en Lcrremoder ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092203003'target=_>, 1859, Almindelig Norsk Lovsamling</a><h4>5429</h4>ansees nodvendige eller gavnlige for Fattig-Vesenets Oiemed . 4 t Skolegangens Oiemed bor vere : at Vernene ved deres Sjels- og Legems-Kreftcrs rigtige Udvikling og Anvendelse kunne dannes til oplyste , gode , arbeidsomme Mennester og Borgere , at de tll den Ende tidligen hindres fra Lediggang og Laster , og venncs til <b>Sedelighet</b> » , Orden og Nenlighed ; at de anferes til saadan Kundstab , som er nedvendig og gavnlig fer Mennesket og Borgeren i Almindelighed ; og at de undervises og eves i saadant Arbeide , hvorved de i Fremtiden , enten i Mangel af andet Nerings-Arbeide kunde fortjene deres Underholdning , eller nyttigen beskjeftige sig i ledige Timer . 42. Undervisningen i Skolerne stal altsaa vere delt imellem Vldenstab , Haand-Arbeide og nyttige Legemsøvelser. 43. Ved Videnstab forstaaes ikke her det , som henhorer til blot Lerdom , eller blot spekulativ Kundstab i egentlig Forstand , men det , som for den menige Almue er fornedent, fatteligt og brugbart , saasom : Lesning , Skrivning , Regning , Religion og Moral, almindelig Kundsiab om Naturen , dens Indretninger og Virkninger ; om borgerlige Indretninger og Pligter , om de vigtigste Sundhcds-Regler , om <b>Fedrelandets</b> Forfatning og Veliggenhed og deslige , alt efter korte og simple Lerebeger , som dertil kunne velges, og med Hensyn til den Forskjel , som ber gjeres i Undervisning for Drenge og <b>Piger</b> . Ogsaa serges for , at Bornene anferes til rigtig og harmonisk Sang , iser Choraler og gode Folkesange . 44. Haond-Arbeidet ber vere af det Slags , som i det daglige Liv kan for Almuens Folk vere almindeligt og gavnligt , og som for Vernene efter Forskjellighet » af Kjon , Alder og Krefter findes passende , samt hvortil i Skolebygningerne kan vere Leilighed og Rum , saasom : at pille Bomuld , at karde Uld og Bomuld , at plukke og spinde Uld , Bomuld , Her , Hamp og Blaar , at tvinde , haspe , vinde , spole , at binde og strikke , at kniple , at lappe paa Kleder , at stoppe Stremper , at knytte Fiskegarn, at veve Bendler og uldne Baand , at gjore Hegter og Fistekroge , stette Kurve og Hatte , at forferdige allehacmde Legetei og andet Nurnberger-Kram , ug andet mere ; og for <b>Pigerne</b> tillige , at tildanne og sy Linned og Kleder m . v. Ved Arbeidets afverlende Afdeling sees blandt andet ogsaa derpaa , at Legems-Krefterne styrkes tilligemed eves . 4. ? . Til at give Undervisning i Haand-Arbeide ansettes ved hver af Fattig- Vesenets Skoler en Leremoder , som anvises den fornedne Plads i Skolebygningen ; og forsaavidt Leremoderen ei kunde gjore det , soger Direktionen at forene sig med een eller stere Arbeidere , som maatte besidde de fornedne Ferdigheder , om visse Timer daglig enten hjemme hus sig selv eller i Skolebygningerne at give Undervisning i Haand-Arbeide . 4 V . Legems-Ovelser ber vere af det Slags , som kunde bidrage til den Styrke , Smidighet» og Behendighet » , som i den arbeidende Klasse ere nodvendige og nyttige . Drengebornene ber om Sommeren ofte feres ud at bade sig i Langen , og tillige gives Undervisning i at svemme . For at dette med Sikkerhed kan ste , treffes de fornodne Foranstaltninger. Til at undervise i Legemsouelser antages helst en duelig Under-Ofsicier , som er oplert i Gymnastiken , da han tillige kunde eve de eldre Drengebern i at erercere. 47. Ingen Understettelse fra Fattig-Vesenet gives til Underholdning for <b>fattige</b> Vern , uden under det udtrykkelige Villaar , at Bornene stulle stadigen sege den Undervisnings og Arbeids-Skole , hvortil de henvises ; og findes Almisse-Lemmer eller Andre , som ved Fattig-Vesenet ere indstrevne , forsemmelige i at tilholde deres Bern at soge Skolerne , eller de selv afholde dem derfra , advares de forst , og nåar Advarsler ei frngte , ansees de efter § lUO . Er det Vernenes egen Skyld , at de udeblive , ansees de medde Straffe , som Skole Reglementerne nermere for saadan Forseelse fastsette . 48. Fra sormeldte <b>offentlige</b> Skolegang kan ug bor ikke , Sygdoms-Tilfelde undtagen , ste Fritogelse , med mindre Direktionen finder gyldige Grunde til saadant at tilstede for en Tid ; men i alle Tilfelde , hvor Undtagelse saaledes tilstedes , ber omhyggeligen serges for , at de Bern , som i den bestemte Tid ikke mede i Skolerne , dog ikke savne den Undervisning og Anfersel til Arbeide , som er Skolegangens Oiemed . 4 » . Naar <b>Fattige</b> , der have Bern , som ber sege Skolen , bo saa langt fra den nermeste Skole , at Vernene tilvisse Tider af Aaret ikke stadigen funne indfinde sig der , ferger Direktionen , saa vidt muligt , for at disse Familier eller blot Vernene flyttes ind til Bopele , som ligge beleiligere , iltt . For saa vidt det er gjerligt , serges fur , at Drengenes og <b>Pigernes</b> Undervisning adstilles , saaledes , at nåar Drengene undervises af Skolelereren , <b>Pigerne</b> anfores af Leremoderen til Haand-Arbeide , og omvendt ; og nåar Saadant er forbundet ^ ned for megen Vanskelighet » , Drenge og <b>Piger</b> holdes saavidt muligt i serskilte Verelser . ilt . Hvor tidlig om Morgenen Vernene stulle mode i Skolerne , hvorledes Tiden , medens de ere der , inddeles , imellem afverlende gradvis fremgaaende og efter deres Evner afpasset Undervisning i Kundstab og Anfersel eller Ouelse i Arbeide , med videre ; hvilke Lere-Methoder og Lerebeger stulle indferes , for rigtigen at udvikle deres Evner , og Veilede mere til Selvtenken end til Udenad-Lesning ; hvorledes Bornene fordeles i Klasser: dette bliver i Reglementer for Skolerne i Almindelighed eller for hver iser efter

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010022412001'target=_>, 1845, Kongelige Rescripter, Resolutioner og Collegial-Breve for Norge</a><h4>2495</h4>ning l Hovedet , dansk ' ) Grammatik og Stiil-Bvelser , Forstands-Bvelser for de yngre Bern efter Rochows Vernev « n , saa og Ovelse i god og harmonisk Sang ; desuden catechiserer han hver Sendags Gftermiddag i Kirken om Sommeren , hvilket , under Navn af Aftensang , hidtil har veret Klokkerens Pligt . § 5 . Alle <b>fattige</b> Bern skulle have fri Adgang til denne Skole . Ved <b>Fattige</b> forstaaes ikke alene , de Born , som enten selv eller igjennem deres Foreldre have Understøttelse af Byens Fattig-Casse , men ogsaa de , hvis Forceldre , uden at nyde saadan Understøttelse , dog maatte ansees trcengende til fri Skolegang for deres Born . De Bvige , som ville sege denne Skole , betale Enhver 1 Rd . qvartaliter . Z 6. Lereren ved Real-Skolen , hvori Ingen antages , uden at kunne lcese reent og ferdigen inden i Vog , men ellers ethvert Byens Barn , hvis Forceldre kunne udrede Skolelsnnen , skal udstrcekke sin Underviisning , foruden til de i den anden Skole opgivne Kundstaber , ogsaa til et mere omfattende Vekjendtstab med Historien og Geographien samt <b>Fedrelandets</b> Statistik og det engelske Sprog , som nermest fornedent for Stedets Handlende ; endelig prediker han til Heimesse hver anden Sendag og Helligdag i Sogneprestens Fravcerelse ved Anneret , hvilket forhen har veret paalagt Klokkeren . § 7. Lesetimerne kunne uere fra iste Marts til iste November daglig ser for hver af Skolerne , nemlige tre om Formiddagen og tre om Eftermiddagen , men fra iste Nov . til Iste Marts fem Timer , tre om Formiddagen og to om Eftermiddagen , hvis Anvendelse og Inddeling , i Henseende til de forskjellige Videnssaber , nermere kan bestemmes ved en speciel Instrur , der tillige skal indeholde Regelen i Henseende til Skolernes Orden og Disciplin . De evrige Timer overlades Vernene til Foreldrenes Omhu ; men dog ber der , hvad de <b>fattige</b> Born angaaer , saavidt muligt ferges for , at de i een eller to af deres Fritimer kunne undervises og sves i et eller andet nyttigt Haand-Arveide , for deraf at heste Fordeel i Fremtiden , og tillige afholdes fra Lediggang og de deraf ftydende Laster . § 5. Lereren maa ingen Frihedsdage tilstede , Lpverdags Eftermiddag undtagen , og i Julen otte Dage fer Heltiden , indtil ferste Mandag efter Nytaar , Dimmel-Ugen indtil ferste Onsdag efter Paaske , fra Fredagen for Pintsedag indtil Onsdagen derefter; ei heller maae Vernene nogensinde udeblive af Skolen , uden at Foreldrene tilmelde Lererne Aarsagen . § 9. Hvert Halv-Aar skal holdes <b>offentlig</b> Gramen i begge Skoler , hvortil Vernenes Foreldre og enhver anden Tilherer skal have Adgang , ligesom det og stedse stal staae Foreldrene frit for at bivaane Forelesninger for Ungdommen . § 40. Vernene , i begge Skoler inddeles i Classer , og Lererne stulle vere forpligtede til , hver for sin Skole , at holoe en Protocol , hvori de Unges Forhold , saavel paa Kundssabernes som <b>Sedelighedens</b> Bane , dagligen anferes ; og stal denne Protocol fremlegges offentligen til Alles Eftersyn ved de bestemte Nxamin » . Desuden skal ethvert af Vernene hver Leverdag i en serskilt Bog gives et skriftligt Vidnesbyrd fra Lcerercn om deres Flid og Forhold , hvilken Bog de Mandags Morgen stulle bringe tilbage paa Skolen med Foreldrenes Paategmng , at Bogen har veret dem foreviist . § Begge Skoler stulle staae under en Commission , som , foruden de sedvanlige Infpectorer , stal bestaae af et Medlem af Fattig-Commissionen , der i Serdeleshed har at paasee de fattigste Berns og Fattig-Casfens Tarv , samt tre af Byens Borgere , der nermere velges paa visse Aar , uden derfor at fritages for andre Byens Ombud, og hvis Pligt det bliver , i Forening med Inspecteuren at vaage over Skol « nes Tarv og Orden , samt verelviis indcassere , og i beherig Tid udbetale saavel de Penge , der tilkomme Lererne af de betalende Foreldre , som og de Bidrag , en edelmodig Subscription maatte skjenke den anden Skole , og ved hvert Aars Udgang fremlegge <b>offentligt</b> Regnstab for Pengenes Anvendelse til Enhvers Efterretning . ' 8 12. Lereren ved den anden Skole

<h4>5957</h4>aer og Virkninger ; om borgerlige Indretninger og Pligter , om de vigtigste Sundheds-Regler , om <b>Fedrelandets</b> Forfatning og Beliggenhet » og deslige , alt ester korte og simple Lcerebeger . som dertil kunne vcrlges , og med Hensyn til den Forssje ! , som ber qjores i Underviisning for Drenge og <b>Piger</b> . Ogsaa sorqes for , at Vernene anfores til rigtig og harmonisk Sang , iscer Choraler og qo ^ e Folkesauge . § 44. Haand-Arbeidet bor vcere af det Slags , som i det daglige Liv kan for Almuens Folk verre almindeligt og gavnligt , og som for Bomene ester Forskjellighet , af Kjon , Alder og Krcefter findes passende , samt hvortil i Skolebygningane kan verre Leilighed og Rum , faasom : at ville Vomuld , at karde Nld og Bomuld , at plukke og spinde U ! d , Vomuld , Hor , Hamp og Vlaar , at tvinde , haspe , vinde , spole.at binde og strikke , at kniple , at lappe paa Klceder. at stoppe Stremper , at knytte Fissegarn , at vceve Bcendler og uldne Baand , at gjere Hcegter og Fistekroge , flette Kurve og Hatte , at forfcerdige allehaande Legetoi og andet Nurnberger-Kram , og andet mere ; og for <b>Pigerne</b> tillige , at tildanne og sye Linned og Klceder , m . v. Ved Arbeidets afverlende Afdeling sees blandt andet ogsaa derpaa , at Legems-Krcefterne styrkes tilligemed oves . § 45. Til at give Underviisning i Haand-Arbeide anscettes ved hver af Fattig-Vcesenets Skoler en Lceremoder , som anvises den fornodne Plads i Skolebygningen ; og forsaavidt Lceremoderen ei kunde gjere det , soger Directionen at forene sig med een eller flere Arbeidere , som maatte besidde de fornodne Fcerdigheder , om visse Timer daglig enten hjemme hos sig selv eller i Skolebygningerne at give Underuiisning i Haand-Arbeide . § 46. Legems-Vvelser bor vcere af det Slags , som kunde bidrage til den Styrke , Smidighed og Behcendighed , som i den arbeidende Classe ere nodvendige og nyttige . Drengebornene ber om Sommeren ofte fores ud at bade sig i Laugen, og tillige gives Underviisning iat svemme . For at dette med Sikkerhet» fan siee , trceffes de fornedne Foranstaltninger . Til at undervise i Legemsøvelser antages helst en duelig Under-Officeer , som er oplcert i Gymnastiken , da han tillige kunde eve de celdre Drengeborn iat erercere . Z 47. Ingen Understettelse fra Fattig-Vcesenet gives til Underholdning for <b>fattige</b> Born , uden under det udtrytkelige Vilkaar , at Vernene skulle stadigen soge den Underviisnings- og Arbeids-Skole , hvortil de henvises ; og findes Almisse- Lemmer eller Andre , som ved Fattig-Vcesenet ere indssrevne , forsommelige i at tilholde deres Bern at soge Skolerne , eller de selv afholde dem derfra > advares de ferst , og naar Advarster ei frugte , ansees de efter 8 iNI ) . Gr det Vernenes egen Skyld , at de udeblive , ansees de med de Straffe , som Skole- Reglementerne ncermere for saadan Forseelse fastscette . Z 48. Fra formeldte <b>offentlige</b> Skolegang kan og bor ikke , Sygdoms-Tilfcelde undtagen , siee Fritagelse , med mindre Directionen sinder gyldige Grunde til saadant at tilstcede for en Tid ; men i alle Tilfcelde , hvor Undtagelse saaledes tilstcedes , ber omhyggeligen sorges for , at de Bern , som i den bestemte Tid ikke mede i Skolerne , dog ikke savne den Undervisning og Anforsel til Arbeide , som er Skolegangens Oiemed . § 49. Naar <b>Fattige</b> , der have Born , som ber soge Skolen , boe saa langt fr « den ncermeste Skole , at Vernene til visse Tider af Aaret ikke stadigen kunne indfinde sig der , ferger Directionen , saa vidt muligt , for at disse Familier eller blot Vernene flyttes ind til Bopcele . som ligge beleiligere . 8 50. For saa vidt det er gjorligt , sorges for , at Drengenes og <b>Pigernes</b> Underviisning adssilles , saaledes at naar Drengene undervises af Skolelæreren , <b>Pigerne</b> anfores af Lceremoderen til Haand-Arbeide , og omvendt ; og naar Saadant er forbundet med for megen Vanskelighet » , Drenge ° g <b>Piger</b> holdes saa vidt muligt i scersiilte Vcerelser . 8 51. Hvor tidlig om Morgenen Vernene skulle mode i Skolerne , hvorledes Tiden , medens be ere der , inddeles , imellem afverlende gradviis fremgaaende og efter deres Evner afpasset Underviisning i Kundsiab og NNfersel eller øvelse i Arbeide , med videre ; hvilke Lcere-Methoder og Lcerebeger skulle indferes , for rigtigen- at uduikle deres Evner , og veilede mere til Selvtcenken end til Udenad-Lcesning; hvorledes Vernene fordeles i Classer : dette bliver i Reglemen- Vergs Rescr . IV Mnd . 29

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016101748015'target=_>Calman, A.L., 1876, John Ashworths Liv og Virksomhed</a><h4>1455</h4>i <b>moralsk</b> og naturlig Henseende — har udviklet sig , har vakt alle Nationers Forbauselse ; dog gjælder dette fremfor Alt om New-lork , hvis Fremvcrxt er mest vidunderlig . Stadens tre hundrede Kirker , dens Barmhjertigheds- og Velgjorenhedsanstalter, der omfatter og lindrer næsten enhver Art af menneskelig Nod og Elendighed , dens Universiteter, <b>offentlige</b> Bivliotheker , videnskabelige Samlinger, Parker og Paladser , — Altsammen vidner om en forbansende Oplysning , Flid og Arbeidsomhed . Dog , der var en Bygning i tredie Linie , den , som vi først pasferede igjennem , der efter min Formening forklarede Grnnden til al denne Velstand og Storhed ; dette er en umaadelig Komvlex af Bygninger, der indtager et helt Kvartal , sex Etager hsi og oplyst af mere end fire hundrede Vinduer . Bygningen er forfynet med den mest ophoiede Indskrift , der nogensinde har navngivet nogen menneskelig Anstalt , og den lyder „ Bibel-Huset " , altfaa Gudsordets Tempel , Oplagsstedet for Bogen fra Himmelen, — og Himmelens Forordning med Hensyn til den Bog er , at hvssomhelst , der gransker i den og gloeder sig ved den , han stal have Held og Lykke i Alt , hvad han foretager sig . Der gives ikke en <b>Nation</b> paa hele Jorden , der kan fremvise Magen til et saadant Palads , bygget i et saadant Diem ' ed ; men saalcrnge Herrens , Haand kjendes i Folkenes Ophsielse og Fald , er det ikke at undres paa , at Han skulde « re det Folk , der cerer Ham . Men nn kommer det Spsrgsmaal : Er ikke dette prcrgtige Bibelhus en Frugt af „ Maiblomsten " ? Rorte ikke Spiren til dette mægtigeßige sig indeni hin strovelige Farkost , der trodsede Atlanterhavets Bolger og ' forte til disse Kyster en Skare af hellige Moend og <b>Kvinder</b> , hvis Hjerter Herren havde

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009082700002'target=_>Taine, Hippolyte, 1877, engelske Literaturs Historie</a><h4>760</h4>» mod disse Mænd ( det vil sige denne Mand ) , med det fordærvede Hjerte og den besmittede Fantasi , som lave sig et Sæt Anskuelser , der svare til deres sørgelige Opførsel, og gjøre Oprør mod det menneskelige Samfunds helligste Bud , som hade den aabenbarede Eeligion , skjøndt de ikke fuldstændigt kunne rive sig løs fra Troen paa den , trods alle deres Anstrængelser og deres udfordrende Pral , og som arbeide paa , at gjøre de andre ligesaa ulykkelige som de selv , ved at smitte dem med en <b>moralsk</b> Gift , som vil æde sig ind hos dem lige til Hjertet « * ) ; maa hører Præstens Salvelse og den Lærdes Pedanteri . 1 dette <b>Land</b> gjør Pressen Tjeneste som Politibetjent , og den har aldrig gjort det med mere Iver end i dette Øieblik ; og den <b>offentlige</b> Mening understøttede Pressen . I Italien saa Lord Byron flere Gange Gentlemen gaae ud af Stuen med deres <b>Damer</b> , naar man meldte ham . Som fornem Adelsmand og berømt Mand , blev den Forargelse , han vakte , mere bemærket end nogen Andens ; han var a public sinner ; en Dag sendte en Landsbypræst ham en Bøn , som han havde fundet i sin nyligt afdøde Kones Papirer , hun , den rene og troende <b>Kvinde</b> , havde i Stilhed bedet Gud , om at den store Synder maatte omvende sig . Efter fem og tyve Aars <b>moralske</b> Kampe og to Aarhundreders <b>moralske</b> Op ' dragelse , drev det konservative og protestantiske England sin Strænghed og sine Sømmelighedskrav til det Yderste , og ligesom fordum den katholske Intolerance i Spanien , saaledes satte nu den puritanske Intolerance enhver afvigende Mening udenfor Loven . At forkaste det vellystige eller frie Liv , strængt at iagttage Reglen og Sømmeligheden, at ære al guddommelig og menneskelig Myndighed , at gjøre det skyldige Buk , saasnart man nævnede Pitt , Kongen , Kirken , og den bibelske Gud , at føre sig som Gentleman med hvidt Halstørklæde , reglementeret , übøielig

<h4>1030</h4>for alle Slags <b>Samfund</b> , og det uden Lon . En af dem har bygget en Bro paa sin Bekostning , en Anden et Kapel , en Tredie en Skole ; Uere af dem oprette Laanebibliotheker med opvarmede og oplyste Værelser , hvor Bønderne om Aftenen finde Aviser , Spil og The for godt Kjeb , kortsagt hæderlige Adspredelser , som drage dem bort fra Værtshuset og Brændevinet . Mange af dem holde Foredrag eller læse op , deres Søstre eller Dettre holde Søndagsskoler, i det Hele give de paa deres egen Bekostning de Uvidende og de <b>Fattige</b> Ketfærdighed , Bestyrelse og Kundskaber. Jeg har seet En , der eiede tredive Millioner , som om Søndagen i sin Skole lærte Smaapiger at synge . Lord Palmerston stiller sin Park til Skytteforeningernes Kaadighed, Hertugen af Marlborough aabner sin hver Dag for Publikum , idet han » beder « ( det er det Ord , han brager ) » de Besogende , om ikke at edelægge Grønsværet « . En grundfæstet og stolt Pligtfølelse , en virkelig Almenaand , en stor Forestilling om , hvad en Gentleman skylder sig selv , giver dem den <b>moralske</b> Overlegenhed , som berettiger til at fore an ; man har rimeligvis siden Oldtidens græske Borgerliv ikke seet en Opdragelse eller en Stilling , hvor Menneskets medfødte Adel fik en sundere og fuldstændigere Udvikling . De ere kort sagt fødte øvrighedspersoner og Beskyttere , de staae i Spidsen for de store Foretagender, hvor man maa sætte Kapitaler paa Spil , de lede al Velgjørenhed , alle Forbedringer , alle Keformer , og med Æren af at fore an , tåge de ogsaa Byrden . Thi man maa lægge Mærke til , at de , i Modsætning til Adelen ellers , ere oplyste og frisindede , og at de gaae i Spidsen for den <b>offentlige</b> Udvikling , og ikke bagefter den . Det er ikke fine Selskabsmennesker , som vore Marquier fra det attende Aarhundrede ; en Lord besøger sine Fiskerier , studerer Læren om flydende Gjødning , taler med Kyndighed om Ost , og hans Søn er ofte dygtigere end hans Forpagtere

<h4>1306</h4>Man seer , at dette er en ny Art Hykleri ; Lasterne skifte hvert Aarhundrede sammen med Dyderne . Den praktiske Aand er ligesaavel engelsk som den <b>moralske</b> Aand ; dette Folk har handlet , arbeidet og regjeret sig selv saalænge , at det tilsidst har faaet Lyst og Evne til Forretninger ; det er derfor de betragte os som Bern og Galninge . Naar denne Retning overdrives , ødelægger den Fantasien og Følelsen , man bliver en Regneraaskine, i hvem der kun lever Kjensgjerninger og Tal , man nægter Aandens Liv og Hjertets Glæder , man seer ikke andet i Verden end Indtægt og Tab , man bliver haard , barsk , begjærlig og gjerrig , man behandler Menneskene som Redskaber , og saa kommer der en Dag , hvor man er bleven lutter Handelsmand , Bankier og Statistiker , og hvor man ikke længere er Menneske . Dickens har givet mangfoldige Skildringer af den kolde praktiske Mand : Ralph Nickleby , Scroogs , Antony Chuzzlewitt , Jonas , Alderman Cute , Mr . Murdstone og hans Søster , Bounderby , Tom Gradgrind , de findes i alle hans Romaner . Nogle have faaet Præget ved Opdragelsen , andre have det fra Naturen af , men de ere alle modbydelige , thi de gjøre det alle til deres Opgave , at spotte og tilintetgjøre Godheden, Medfølelsen , Medlidenheden , de uegennyttige Følelser , de religiøse Stemninger , Fantasiens Begeistring , kort sagt , alt det Skjønne hos Mennesket . De undertrykke Børaene , de slaae <b>Kvinderne</b> , de sulte de <b>Fattige</b> , de forhaane de Ulykkelige . De bedste af dem ere glatte Jernautomater , der regelret udfore de Pligter , Loven paalægger dem , og ikke vide , hvormegen Smerte- de paadrage andre . Den Slags Mennesker findes ikke i vort <b>Fædreland</b> ; deres

<h4>1339</h4>Saa felger det da af sig selv , at Dickens forherliger dem . Det er ikke noget nyt i Frankrig ; Eugen Sue ' s Bomaner have givet os mere end eet Exempel derpaa, og det er en Paastand , der naaer tilbage til Rousseau, nåen under den engelske Forfatters Hænder har den faaet en særlig Styrke . Hans Helte ere beundringsværdigt hensynsfulde og opoffrende , der er ikke andet simpelt ved dem end deres Udtale , ellers ere de lutter Adel og delmodighed. Vi see én Gjøgler forlade sin Datter , hans eneste Glæde , af Frygt for at skade hende i Noget . En ung <b>Kvinde</b> , der elsker en Mand , som elsker hende igjen , opoffrer sig for at frelse denne Mands uværdige <b>Hustru</b> ; Handen døer , men hun vedbliver af ren Selvfornægtelse at pleie denne nedværdigede Skabning . En <b>fattig</b> Fragtmand , der troer , at hans <b>Kone</b> er ham utro , erklærer hende med hei Køst for uskyldig , og istedetfor Hævn tænker han kun paa at overvælde hende med Ømhed og Godhed . Der er ifølge Dickens ingen , der saa levende som dem føler , hvor stor en Lykke det er at elske og blive elsket , og hvor ren en Glæde Familielivet kan give . Der er ingen , der har saa megen Medlidenhed med disse stakkels , vanskabte og svage Væsener , som de saa ofte bringe til Verden , Jog som kun synes at fødes for at døe . Der er ingen , der har en sundere og mere übøielig <b>moralsk</b> Sands , ja , jeg maa endogsaa tilstaa , at disse Helte hos Dickens ere ulykkelige nok til at ligne de vrede Fædre i vore Melodramer . Da den gamle Peggotty faaer at vide , at hans Søsterdatter er blevet forført , giver han sig paa Vei med Stokken i Haanden , og drager gjennem Frankrig , Tydskland og Italien , for at finde hende og føre hende tilbage til hendes Pligt . Men fremfor alt have de en engelsk Følelse , og som mangler hos os , de ere nemlig Kristne ; det er ikke blot som hos os <b>Kvinderne</b> , der tåge deres Tilflugt til Forestillingen om et andet Liv , ogsaa Mændene tæuke derpaa . I dette <b>Land</b> , hvor der er saamange Sekter , og

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010020112014'target=_>Sundt, Eilert, 1870, Om fattigforholdene i Christiania</a><h4>8241</h4>Stadsfysikus Bidenkap : Det var , forsaavidt jeg har forstaaet Koniiteens Mening , om Sygepleiens Organisation , jeg skulde meddele nogle Oplysninger. Endel saadanne findes allerede i Sundts Bog om Fattigforholdene i Christiania . Der er taget med baade Beretning om Lægetilsynet og om Sygehusforpleiningen ( dog mindre om denne Sidste ) samt om de almindeligste Sygdomme blandt de <b>Fattige</b> og om forskjellige Ting , som jeg altsaa ikke behøver at gjentage . Ligeledes vil der i den Forklaring , som er udarbeidet af en Komite inden Fattigkommissionen , hvoraf jeg var Medlem , og som blev afgivet den 30 te November 1869 , findes en kort Fremstilling af Organisationen af Sygepleien . Jeg skal derfor henvise til disse trykte Dokumenter , som jeg vil supplere med , hvad jeg har at sige , som da væsentlig er nogle yderligere statistiske Oplysninger . Angaaende Organisationen af den <b>offentlige</b> Sygepleie i sin Almindelighed her i Christiania skal jeg oplyse , at denne er paabyrdet , kan jeg gjerne sige , Fattigkommissionen i en meget videre Udstrækning , end man kunde forudsætte efter Loven og efter Forholdene paa andre Steder hertillands. Fattigvæsenet har nemlig overtaget ikke alene Behandlingen og Forpleiningen af <b>fattige</b> Syge i Hjemmet og paa Sygehusene , men ogsaa Indredningen og Driften af Epidemi- Lazarether , og andre Sygehuse , som er bestemt for specielle Sygdomme , der maa skjøttes af det <b>Offentlige</b> , ikke paa Grund af at de Syge ere <b>Fattige</b> , eller ikke væsentlig ialfald af den Grund , men fordi Sygdommen er faretruende for <b>Samfundet</b> . Alle epidemiske og smitsomme Sygdomme har altsaa hidtil været behandlet af de samme Funktionærer, som har med Fattigpleien forøvrigt at gjøre , — tildels med extraordinær Hjælp rigtignok , — og for Fattigvæsenets Regning . Kun under store Epidemier , som Kolera , og leilighedsvis under andre Epidemier er der gjort Undtagelser herfrå . Blandt disse Sygdomme , Fattigvæsenet har taget sig af . er ogsaa en , som mindst af alle egentlig synes at burde indgaa under Fattigadministrationens Omraade , det er de Foranstaltninger , som træffes mod den veneriske Sygdom . Endel af Udgifterne ved disse bæres af Bykassen , nemlig forsaavidt Visitationen af de prostituerede <b>Kvinder</b> angaar ; men det er ogsaa Alt . Forpleiningen paa Sygehus af dem , som tvungent eller frivilligt indlægges der , foregaar for Fattigvæsenets Regning . Selve Administrationen af Prostitutionsvæsenet henligger vistnok under Sundhedskommissionen og det almindelige Politi ; men der benyttes dog dertil ogsaa Funktionærer , som lønnes af Fattigvæsenet , nemlig Assistenterne ved Politikammeret for Fattigvæsenets Anliggender , saaat Fattigvæsenet ogsaa er blandet ind i denne Sag . Desuden har Fattigvæsenet overtaget Omsorgen for de Sindssyge , saaat det ikke alene betaler for deres Forpleining , men ogsaa seiv holder et Sindssygeasyl, som administreres af Fattigkommissionen . Det er tydeligt , at paa denne Maade overføres der paa Fattigkassen en stor Del Udgifter , som i Landdistrikterne ikke bæres af vedkommende Kommune , men af Amtskomnmnen, ialfald for en større Del . Alle de Sindssyge , som er i den Tilstand , at de tiltrænger en fra den sædvanlige Fattigforsørgelse forskjellig Behandling , underholdes nemlig efter Loven af Amtskommunen for 4 / s Dele ; her underholdes disse helt ud af Fattigkassen . Det samme er Tilfældet med de epidemiske Sygdomme i Landdistrikterne , naar Sygdommen er erklæret for Epidemi. De Ting , som her er taget under Fattigvæsenet , gjør altsaa , at det Hele bliver et temmelig vidtløftigt Administrationsfeldt . Forpleiningen af de <b>fattige</b> Syge foregaar dels i Hjemmet , dels i private Familier , hvor de Syge udsættes , dels i Sygehusene og for en ganske ringe Del i Arbeidsanstalten . De Sindssyges Forpleining sker enten i Hjemmet eller ved Udsættelse i privat Forpleining eller i Kommunens Sindssygeasyl.

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112201034'target=_>Taine, Hippolyte, 1874, engelske Literaturs Historie</a><h4>2810</h4>med Døden for Øie og vidende Baalet rede . Hans Sundhed var slet , han led af stærk Hovedpine , af Navesmerter, Pleuritis og Steen , men trods dette arbeidede han kraftigt og uophørligt ; han reiste , han skrev , han prædikede ; endnu i sit syv og tredsindstyvende Aar holdt han to Taler hver Søndag , som oftest stod han op Klokken to om Morgenen , baade Sommer og Vinter , for at studere . Hans Veltalenhed var meget simpel og meget virksom , og den var virksom , fordi nan aldrig talte for at tale , men for at handle og gjøre sin Pligt . Hans Instruxsr og Foredrag , blandt andet dem , han holdt for den unge Konge Edward , svæve ikke i Luften som Massillons Foredrag for den lille Ludvig XV , de bevæge sig ikke i den filosofiske , den oversvævende Udviklings rolige Egne ; det er de nærværende Laster , han vil rette og som han angriber , de Laster , som han har seet , og som Enhver kan pege paa ; og han udpeger dem , han nævner Tingene ved deres rette Navn og Menneskene med , han kommer frem med Kjendsgjerningerne og Enkelthederne som en modig Mand , der ikke skaaner Nogen , men glemmer sig selv og vover Alt for at drage Uretfærdigheden for Lyset oghjælpe den Forurettede . Hvor altomfattende hans <b>Moral</b> , og hvor gammel hans Text end er , han anvender den dog paa de Samtidige , paa sine Tilhørere, snart paa de Dommere , der ere tilstede « de Herrer Fløielsklædte " , som ikke vil66 FIV6 de <b>Fattige</b> Lov at tale til sig , som i hele tolv Maaneder kun have givet den og den <b>Kvinde</b> een Gang Audiens , og som holde en anden stakkels <b>Kvinde</b> i Fængsel , uden at ville tåge mod Kaution; snart paa de kongelige Kasserere og Leverandører , hvis Tyverier han opregner , hvem han giver Valget mellem  Helvede og det Stjaalnes Tilbagelevering " , og fra hvem nan Daler for Daler vrister de uretfærdigt erhvervede Penge og faaer dem givne tilbage . Han gaaer stadigt fra den <b>almene</b> Uretfærdighed til 66 bestemte givne Misbrug , thi 66 t er Misbruget , der skriger og kræver ikke en dannet Taler men en Stridsmand ; Theologien staaer for ham

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010012603002'target=_>, 1844, Motiveret Udkast til almind. Lov om Fattigvæsenet i Kjøbstæderne og paa Landet</a><h4>2033</h4>Foruden den alinindelige Velgjorenheds-Forening gives der flere Privatforeningcr til bestemte Formaal t . Ex . Foreninger for Borneasyler , Borneskoler , Indusirie-Skoler,Sy^eforpleiningsanstalter, Districtsforeninger for Indretninger for <b>moralst</b> fordcrrve Born , til Forbedring af de fattigste Menigheders Kaar , til Forsorgelse af « Idre ugifte <b>Fruentimmer</b> ^ til Undersiottelse af Huusarme og saadanne <b>Fattige</b> , hvis Kaar og Stilling forbyde dem at soge <b>offentlig</b> Understøttelse , til Hjcrlp for entledigede Straffefanger o . s . v. Disse Privatforeningcr staae meer eller mindre i Forbindelse med den almindelige Velgjorenheds-Forening .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008062004080'target=_>, 1861, Læsebog i Modersmaalet for Skolernes højere Klasser</a><h4>8064</h4>ivente , end Spot og Bebrejdelser over sin daarlige Udenlandsrejse . I Kristianssand opholdt han sig et halvt Aars Tid , og erncrrede sig vcd at undervise i Fransk , hvorved han ved Vinterens Udgang havde opsparet sig en Kapital af 12 Rigsdaler . Alle de Savn og Lidelser han havde dejet paa sin ferste Udenlandsrejse , havde ikke formanet at dcrmpe hans Rejselyst , og med de faa Penge han havde lagt op begav han sig om Foraarct 1706 til England , sammen med en Landsmand , en Student Brix , som var ligcsaa <b>fattig</b> som han selv , men havde en velstaaende Moder i Vaghaandcn . Han gik fra London til Oxford , for der at fortsatte sine Studeringen ved det store bodleyanste Bibliothek . Her tilbragte han omtrent to Aar , sikkert nogle af de behageligste i sit Liv ; thi ikke alene havde han her rigelig Anledning til at tilfredsstille sin umættelige Lcrrelyst ved et af Europas ypperste Universiteter , men han fandt ogsaa trofaste Venner og en interessant og selskabelig Omgangskreds blandt de oxfordste Studerende , af hvem han snart blev meget afholdt paa Grund af sit lystige Lune , som dog altid var parret med en vis Mrbarhed og Moderation . Fra Oxford rejste han 17 N8 til Kjobenhavn , hvor han forsegle at staa sig igjennem ved at holde et Slags Privatforclcrsningcr for Studenterne ; men da dette mislykkedes , maatte han atter tage fat paa Huslcrrerbestillingen . Han blev ansat hos en Etatsraad Winding , hvis unge Son han som Hovmester ledsagede paa en kort Rejse i Tydstland ; senere hos Admiral Gedde , hos hvem han var et Aars Tid , indtil han erholdt fri Bolig og et lidet Stipendium paa Borchs Kollegium ( 1709 ) . Her udgav han sit forste Skrift , en Frugt af hans Studeringer i Oxford: Introduction til de europcriste Rigers Historie . Nogen Tid efter forfattede han et andet historist Vcrrk , som han i Haandstrift overrakte Fredrik den fjerde , og til Len derfor udncrvnte Kongen ham til Professor exti-g , - or6inariuß ( uden Len ) . Imidlertid havde han erholdt et <b>offentligt</b> Rejsestipendium paa 10 N Rigsdaler aarlig i 4 Aar , og dermed begav han sig atter udenlands , paa sin fjerde og storste til Holland og Belgien , hvorfra han gik tilfods til Paris . I denne By tilbragte han halvandet Aar , i Begyndelsen kun beffjeftiget med siue Boger ; senere , da Studerekammeret blev ham for trangt , segle han selskabelig Omgang , var en flittig Tilhorer ved de Lcrrdes Dlsputerovelser , og en ikke mindre opmertsom lagttager af d.t brogede Folkeliv i den store Hovecstad . Men han syntes endnu ikke han havde sect nok af Verden , og da han engang tilfcrldigvis herte at » nan kunde komme til Rom for 20 Daler, stulde ter ikke mere til for at vcrtke N.jsclysten ; han bred overtvert mcd Advarsler og Bclcrnkeligheder og drog asstcd , forst til Marseille , og derfra tilios til Genua . Paa Vejen did var han nwr bleven opkavrct af en algiersk Sersver , og da han var tommen i <b>Land</b> , blev han saa alvorlig syg af en Feber , han havde paadraget sig paa Overrejsen , at han « crnkte han aldrig stulde rejse sig igjen . Han stred dog imod saa godt han tundc , „ satte et haardt Hjerte mod Sygdommen , " og kom sia , saavidt , at ban kunde tomme afstcd igjen til Civil « vecchia , hvorfra han gik tilfods til Rom . Her blev han Vinteren over , og log den ' ra , baade for sill Helbreds Skyld , og paa Grund af sin mislige Pengeforfatning , tilfods tilbage til Paris , over Appeninerne til Turin , derfra over Alperne gjennem Saroyen og Dauphine . Efter nogen Tids Ophold i Paris rejste han til Amsterdam , hvor hans Feber, som havde fulgt ham paa hele Rejsen , med et forlod ham . Fra Amsterdam rejste han over Hamburg tilbage til Kjobenhavn . To Aar efter sin Hjemkomst , i hvilken Tid han udgav „ <b>Moralske</b> Kjerne , eller Introduktion til Natur- og Folkerettens Kundstab , " blev han udncrvnt

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010020412003'target=_>Sundt, Eilert, 1850, Beretning om Fante- eller Landstrygerfolket i Norge</a><h4>1434</h4>Musikdyrker , bereiste nylig paa <b>offentlig</b> Bekostning nogle af vore Fjeldbygdcr for at opfange de eiendommelige Psalmemelodier og verdslige Sange og Slaatter , som der have hjemme . Hvorledes mener man nu han bar sig ad dermed ? Havdc han uden videre som Fremmed anmodet Vsnderfolkene om at synge og spille for ham , skulde de vistnok kun seet paa ham med Forundring eller vendt sig sra ham med Mistro . Derfor sang og spillede han selv for dem forst , og nu flokkede de sig om haln , Mcend og <b>Kvinder</b> , Gamle og Unge , og lyttcde med Henrykkelse; naar saa han holdt inde , saa var det , som om besicegtede Strenge vare slagne an i deres Vryst ; thi nu sang og spillcde de af Hjertens Lyst , medens han horte ester . — Saalcdes maa den , som i et eller andet Stykke snster at hore Almuesmandens Tanker , vide at scette sig i et livligt Verelforhold til ham ; han maa bcere med sig visse Anskuelser og Interesser og meddele dem til Andre , for saaledes at faa se disse meddelsommc igjen . Om Nogcn til Erempel vil soge at gjore sig fortrolig med de religwse Forholde i en Bygd , saa maa han selv vcere religiossindet og vel forfaren i Guds Ord og saa i Samtale med dem , han moder paa sin Vei , vise Lyst og Gave til at underholde sig om alvorlige Ting , ja til at formane og opbygge , og jo mere oprigtig og nidkjcer han lelv er , desto mere ville Folkets Hjerters Tanker aabenbare sig for ham . — Onsker Nogen at skaffe sig Kundssab om , hvordan Fattigvesenet har udviklet sig paa et Sted , hvordan Forholdet er mellcm <b>Fattige</b> og Rige , saa maa han forud have Skjon paa de vrange Vcgreber og Forviklinger , som i dette Forhold let kunne opstaa , og vcere besjelet af Lyst til at se alle Misligheder rettede , og han vil mode Fortrolighed i samme Grad , som han ved sin Tale allerede viser sig at vcere fortrolig med Alt . — Der er , for endnu at anfore et Erempel , en Mislighed, som i mange , ja i de fleste Egne i vort <b>Land</b> klceber ved den lavere Almues huslige Liv , nemlig en hoi Grad af Urenlighed, som navnlig ogsaa af den Grund er saa fordcrrvelig for det <b>moralske</b> Liv , at den maaske mere end noget Andet vedligeholder en bred Klsft mellem de hoicre og lavere Klasser ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008090903001'target=_>Sundt, Eilert, 1852, Beretning om Fante- eller Landstrygerfolket i Norge</a><h4>1438</h4>Musikdyrker , bereiste nylig paa <b>offentlig</b> Bekostning nogle af vore Fjeldbygder for at opfange de eiendommelige Psalmcmelodier og verdslige Sange og Slaatter , som dcr have hjemme . Hvorledes mener man nu han bar sig ad dermed ? Havde han uden videre som Fremmed anmodet Bonderfolkenc om at synge og spille for ham , stnlde de vistnok kun seet paa ham med Forundring eller vendt sig fra ham med Mistro . Derfor sang og spillede han selv for dem forst , og nu flokkede de sig om ham , Mend og <b>Kvinder</b> , Gamle og Unge , og lyttede med Henrykkelse; nåar saa han holdt inde , saa var det , som om bestegtcde Strenge vare slagne an i deres Bryst ; thi nu sang og spillede de af Hjertens Lyst , medens han horte efter . — Saaledes maa den , som i et eller andet Stykke onster at hore Almuesmandcns Tanker , vide at sette sig i et livligt Ver clforhold til ham ; han maa bere med sig visse Anskuelser og Interesser og meddele dem til Andre , for saaledes at faa se disse meddelsomme igjen . Om Nogen til Erempel vil soge at gjore sig fortrolig med de rcligiose Forholde i en Bygd , saa maa han selv vere religiossindet og vel forfaren i Guds Ord og saa i Samtale med dem , han moder paa sin Vei , vise Lyst og Gave til at underholde sig om alvorlige Ting , ja til at formane og opbygge , og jo mere oprigtig og nidtjer han selv er , dcsto mere ville Folkets Hjerters Tanker aabenbare sig for ham . — Onstcr Nogen at staffe sig Kundstab om , hvordan Fattigvesenet har udviklet sig paa et Sted , hvordan Forholdet er mellem <b>Fattige</b> og Rige , saa maa han forud have Skjon paa.de vrange Begrebcr og Forviklinger , som i dette Forhold let kunne opstaa , og vere besjelet af Lyst til at se alle Misligheder rettede , og han vil mode Fortrolighed i samme Grad , som han ved sin Tale allerede viser sig at vere fortrolig mcd Alt . — Der er , for endnu at anfore et Erempel , en Mislighed, som i mange , ja i de fleste Egne i vort <b>Land</b> kleber ved den lavere Almues huslige Liv , nemlig en hoi Grad af Urcnlighcd, som navnlig ogsaa af den Grund er saa fordervelig for det <b>moralske</b> Liv , at den maaste mere end noget Andet vedilgeholder en bred Kloft mellem de holere og lavere Klasser ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081903006'target=_>Kurtz, Joh. Heinr., 1859, Lærebog i kirkehistorien for studerende</a><h4>676</h4>og derfor heller ikke fandt nogen videre Efterlignelse . Derimot» bestrebte de mere velhavende Hedningemcnighcder sig bestandig for at bevise deres Vroderkjerlighed ved Collecter for den fra Begyndelsen af <b>fattige</b> og ved Ulykker ( Hungcrsnod ) endnu mere betrengte Menighed i Jernsalem . — Den gamle Verdens tre sedelige Kreftstader : Foragt for fremmed Nationalitet , <b>Kvindens</b> Nedverdigelse og Slaveriet , bleve overvundne efter Apostelens Ord Gal . 3 , 28 ved snccessiv Verdensfornyelse indenfra ndad uden voldsom Kamp mod bestaaende Rettigheder , og Bevidstheden om at vere Medlem i <b>Samfundet</b> under det ene Hoved i Himlen gjenncmtrengte helliggjsrcnde alle Forhold i det jordiske Liv . — Rigtignok blev ogsaa den christelige Neenhcds Speil allerede i den apostoliste Tid paa mange Maader fordunklet af Rustpletter : Hykleri ( Apostl . Gj . 5 ) og Uenighet ) ( K . 6 ) fremtraadte i enkelte Exempler allerede meget tidligt i Brodermenighedcn, men hiint blev straffet ved en frygtclig alvorlig Gudsdom , denne overvunden i Kjerlighed og EftergivenheK I de rigere Hedningcmenig heder ( Korinth , Thessalonika ) trengte Verdensaanden ind som Ippighet » , Selvsyge , Hovmod etc. , men blev ogsaa her brndt og ndstodt ved apostolisk Formaning og Tngt , ligesom ved de luttrende og rensende Christenforfolgelser . Hvo der gav <b>offentlig</b> Forargelse ved grovelig at krenke den rene Lere eller christelige Vandel og trods Sjelesorgerncs Formaning holdt fast ved sin Vildfarelfe , blev ndstsdt af Menigheden , men den Angrende igjen beredvilligt optagen efter tilstrekkelig Provelse . Et Exempel paa den i saa Fald apostoliste Anordning giver Blodfljenderens Historie i Korinth ( 1. Kor . 5 , 1 ff ; 2. Kor . 2 , 5 ff ; smlgn . ogsaa I.Tim . 1 , 19 - 20 ; Gal . l , B — 9 ; 1. J0h . 2,19 etc. ) .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010011903026'target=_>Becker, Karl Friedrich, 1851, Karl Frederik Becker's verdenshistorie</a><h4>1246</h4>Endnu altid bleve Islcenderne drevne til den sydlige Verden as en nesten instinctmessig Hang til at reise . Det var en Mre , naar Nogen havde reist vidt omkring og knnde fortelle om fremmede Sceder . len gammel islandsk Bog hcdder det :  Z3de og usselt er von Liv , naar vi forblive her paa Vjergene , i Dalene og i Arkenerne og aldrig komme til Andre , eller Andre komme til os . " Og i  Kongespeilet " , et andet Skrift , laser man :  Vil Du vere fuldkommen i Videnskaber , saa lcer alle Sftrog , men iscrr Latin og Welsk , thi disse Sftrog gaae videst . " Der bliver ogsaa en Ingling indforl saaledes talende : » Da jeg nu er i mine bedste Aar , vil jeg som Kiobmand sec andre Lande , thi jeg vover ikke at begive mig til Hoffet , forend jeg forft har lcrrt at tiende andre Folkeslags <b>Scrder</b> . " Islcrndercn Bjorn reiste tre Gange til Rom , tog sin <b>Kone</b> Olufa med til det hellige <b>Land</b> og holdt en noiagtig Dagbog over sin Reise , hvilken Dagbog vi endnu have . Mange af disse Islendere og andre siandinaviske Pilegrime toge Krigstjeneste i Eyden , i Serdeleshed i Constantinopel , og udmerkede sig ved Tapperhed og Kierlighed til Berommelse . Men faa forbleve i Udlandet , de fleste vendte , rige paaWrc og Gods , tilbage til deres Hjem , for der at ende deres Dage * ) . Kierlighed til Poesie havde Islenderne medbragt fra deres Stammcland . De dyrkede denne Knnst med megen Iver , og hos de nordiske Konger see vi islandske Skjalde som Hofdigtere . Ved denne Gave og denne Aandsretning faaer den <b>fattige</b> Ve , der ligesom ligger uden for den historiske Verden , en merkverdig Betydning i Poesiens Historie . De gamle nordiske Sagn , der i lang Tid ligesom de greske havde forplantet sig fra Munv til Mund , bleve nu oftstrevne . De to beromte Samlinger, der ofte blive nevnte under Navn af den e l d r e og v n g r e E d d a , sysselsatte sig fornemmelig med Odin og Gudcrne . Men denne Drift til at forplante Sagnene er ogsaa modnet til Historieskrivning , og den islandske Lagmand Snorro Sturlesen ( d0d1241 ) har efterladt et fortreffelig ! Verk over den nordiske Historie . Noget tidligere forfattcde den danske S a i ' o Gramma ticus ien ziirlig Latin en Historie om sit

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010021803052'target=_>Zschokke, Heinrich, 1857, Huusandagtsbog</a><h4>1019</h4>med sund Fode , ligemeget om den er letter eller ikke , og staffe sin <b>Hustru</b> og sine Born de forste , uundverligste Fornodenheder - — haner endnu ikke <b>fattig</b> . I ethvert <b>Land</b> , enhver Stad og Vy maa Enhver forst settes i den Stilling , at han ved sit Arbeide kan fortjene saa Meget . Den , der ikke kan arbeide , ham tilkommer der Almisse fra Barmhjertighetens Haand . Den , der ikke vil arbeide , bor straffes . Den orkeslose Trygler er en <b>offentlig</b> Tyv , der efterstreber Andres Eiendom og formindster den <b>offentlige</b> Formue ; uden nogen Berettigelse eller Adkomst og uden Tak lever han paa Andres Bekostning .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015012208070'target=_>, 1880, Norge</a><h4>2449</h4>Saasom Bergens Indbyggere ere en Samling af alle Nationer , saa ser man dem meget udi Talemaader , Skik og <b>Sæder</b> at differere fra andre Norske . Og er der ingen <b>Nation</b> , som de mere have udkopieret udi Sprog , Tarvelighed og Arbejdsomhed end den hollandske. Borgerfolk havde for 30 Aar siden deres egen Dragt , hvilken de bestandigen holdt vedlige , uden at rette sig efter fremmede Moder , saa at man paa Klæderne strax kunde kjende en Borger-Kone eller Dotter fra Folk af anden Stand ; thi Magistrats- Personer , Gejstlige og Folk af militaire Stand fulgte de almindelige Lande-Moder og alle de Forandringer , som de ere underkastede , saa at man derfor kunde anse hele Ny-Kirke- Sogns Folk og det øvrige af Stranden i visse Maader som en anden <b>Nation</b> og Staden bestaaende af tvende Partier , hvoraf det ene gjorde Commentarier og Gloser over det andets saa vel Klædedragt og <b>Sæder</b> som Talemaader. Saaledes bleve i min Ungdom Borgerfolk af de andre spotvis kaldne Peter af det Hollandske Ord Petie eller Faster , efterdi de kaldte deres Faders Søstere og Morbrødre Peter og Ohmer . Borgerfolk forsømmede ikke paa deres Side at gjøre Critiquer over de andre saakaldne fornemme Folk , hvilke de ansaa som <b>fattige</b> og hofifærdige , og til Vederlag kaldte dem Top-Høner , efterdi de bare Sætte og Fontanger , som var en contrebande Klædedragt paa Stranden , hvor Borger-Koner samt <b>Piger</b> og Døtre bare Huer eller rettere Hætter af en særdeles Fagon , hartad gjorte som de Straahætter, <b>Fruentimmeret</b> bruger om Sommeren paa Landet . De samme kaldes gemenligen Skur og bestode af fint Lærred . Mandfolkenes Dragt var og hel simpel , saa at de fleste Borgere saa ud som Skippere , bærende Kjoler uden Haandklapper .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112201009'target=_>Prytz, A.F.W.J., 1880, Livsbilleder fra Kristi Kirkes Historie</a><h4>866</h4>usedvanlige Styrke , der gjorde ham sterk nok til at tåge fat paa den Kjemveopgave , der nn forelaa ham . Fsrend vi imidlertid gaa over til at paavise , hvorledes Kalvin virkede for sin Opgave , maa vi betragte lidt nermere Kalvins Anstuelfer om Kirken og det Kirkeregimente , han fik indfsrt i Genf . Kalvin tilstod Kirken en stsrre Auktoritet end de andre Reformatorer . Han siger :  Kirken er vor Moder . Allerede denne enkelte Benevnelse viser , hvor nyttigt det er at lere den at kjende . Thi vi kunne ikke anderledes indgaa til Livet , end naar den fsder os af sit Skjsd , nerer os ved sin Barm og holder os under sit Opsyn og sin Beskyttelse , indtil vi , befrieoe fra dette dodelige Liv , blive Engle lige . " Denne Kirke har — lerer han videre — et af Gud indsat Lereembede , hvilket de Troende ere forpligtede til at vise Lydighed . Den har Ret til at optrede lerende . Synoden , som representerer den , og som bestaar af Geistlige og Mldste , yher den bedste Borgen for den rigtige Udleggelse af den hellige Skrift . Der tilkommer Kirken Ret til at give Love og haandheve Kirketugt . Intet <b>Samfund</b> kan beståa uden Orden og Tugt . Leren er Kirkens Sjel , Tugten er at sammenligne med Nerverne , som formidler Forbindelsen med de forskjellige Lemmer og holder denne Ordning fammen . Den er en Tsile for de Onde , en Spore for de Gode , et faderligt Ris . Den gjennemlsber forskjellige Grader , fra Privatformaning til <b>offentlig</b> Anklage og til Udelukkelse fra den hellige Nadvere . Forat den kan vere retmessig , maa den ikke haandheves af de Geistlige alene . Den saaledes organiserede Kirke er i sit Vesen strengt forfkjellig fra Staten . Denne arbeider mod rent jordiske Formaal , medens Kirken streber mod et himmelsk Maal . Derfor har den ingensomhelst Ret til at herske over Kirken eller til at blande sig ind i rent kirkelige Sager . Kalvin ssgte nu at benytte den Seier , han havde vundet , til at praktisere sine Theorier . Straks efter sin Ankomst erklerede han , at han ikke kunde bestyre sit Embede , uden at der blev oprettet et velordnet Presbyterium , forbundet med kirkelig Tugt . Derpaa gjorde han Udkast til en Kirkelovgivning , som den 20 de November i selvsamme Aar paa en hsitidelig Maade blev antaget af det samlede Borgerskab . Ved denne Lov blev Stadens kirkelige Inddeling fastsat , Antallet af G ei f t lige bestemt , deres Forretningskreds anvist og Valget af dem ordnet paa den Maade , at de bleve valgte af de andre Prester ; men dette Valg skulde bekreftes af Naadet og Menighederne . Ved Siden af Geistligheden anordnede han LEldste ( Presbytere ) , fom bleve udpegede af Presteskabet , men hvis Valg ligeledes blev stadfestet af Menighederne. Konfistoriet , en Domstol , der havde at vaage over Sedernes Renhed og at forvalte Kirkens Formue , blev valgt af og blandt Medlemmer af det borgelige Raad ( mod Kalvins Anste , der krevede det sammensat af Geistlige og sEldste ) . Til de forannevnte Menighedens Embedsmend , Prester og Mldste , kom endelig Di a <b>kone</b> r ne , Medhjelpere i Sjelesorgen blandt <b>Fattige</b> og Syge . Det var et jernhaardt Regimente , der nu blev lagt paa Genserne . Konsistoriemedlemmerne turde betrede hvilketsomhelst Hus til enhver Tid . Konfirmationen fandt Sted i det 16 de Aar ; Konfirmanderne bleve noie prsvede , forinden de stedtes til den fsrste Altergang . Ingen Daab skulde finde Sted i Husene , men i Kirken for den forsamlede Menighed . Konsistoriet havde Ret til at tale med ved Indgaaelsen af Mgtefkab . Der fandtes kun kirkelig Vielfe . Ingen turde klede sig  over sin Stand . " Den selflabelige Omgang blev sat sine Skranker : Antallet af Gjester og Retter blev bestemt !

<h4>1084</h4>og at der var mange , mange Sjcele , som havde fslt Mangelen , og som med Glcede vilde gjsre Sit til for at raade Bod derpaa . Et stort Antal Prester saavel i hsiere som lavere Stilling udtalte sil : fulde Billigelse af , hvad der var udtalt i Speners ? ia 6 e3i6Sria ( Fromme Vnste ) , og bsde ham broderlig Haanden til fcelles Arbeide mod det store Maal , fremme af en livskraftig Kristendom . At der paa den anden Side udkom alskens Smcedestrifter og onde Rygtcr , var en Selvfslge , aldenstund Spener saa uforbeholdent gik los paa Skinkristendom og dsde Former , og for hans eget Vedkommende havde denne Forfslgelse den gode Virkning , at han gav mere Agt paa sig selv og gik forsigtigere frem , og saalangt fra at bersve ham Modet , gjorde det ham kun trsstigere . Saa blev han nu ogsaa glcrdet ved mangt et Budskab om , at den udftrsede <b>Sced</b> havde begyndt saa smaat at spire .  Ak , at det , som vi se her og der , maatte verre Skud , der bebude det snart kommende Foraar ! " skrev han da . Den nye Bevcegelse spredte sig omkring — fra Hus til Hus , fra Familie til Familie . Den greb Menneskene bande ide lave Hytter og i Slottene . Over det hele Tyskland gik der alskens Rygter om Spener og hans Lige sindede , — at de havde indfsrt fuldstcendigt Acrllesskab i Gods , — at i deres Forsamlinger baade Mamd og <b>Kvinder</b> prcedikede , — at alle Speners Til hcengere istedenfor den sedvanlige Dragt bare Linklceder , — at <b>Kvinderne</b> forssmte Husholdningen og i Sammenkomsterne sammen med sine Veninder fortalde alflens Lcetkerier , — at Spener var afsat fra sitEmbede — og lignende . Men det blev ikke kun ved disse onde Rygter . Speners Svoger , Horbius , blev afsat , fordi han hensynslsst prcrdikede og virkede i Speners Aand . Spener selv truedes med Anklage , fordi han roste mystiske Skrifter , der i sin Udtryksmaade fjcernede fig fra det orthodokse Sprog . Man paastod desuden , at han ivrede for Helliggjsrelsen paa Retfcerdiggjsrelsens Bekostning , og at han bersvede de Anfcegtede al Trsst . Men Spener vedblev rolig at arbeide videre uden at agte paa alt dette . Som et Middel , hvorved der kunde raades Bod Paa den herskende Ugudelighed blandt de Kristne , anbefalede han , at de alvorlige Menighedslemmer skulde slutte sig sammen til mindre <b>Samfund</b> , hvorfra da en Livets Aand vilde blcrse ud til de dsde Menighedslemmer . De Orthodokse havde kun tcenkt paa Bevarelsen af den reneLcere , de havde holdt Vagt paa Guds Stads Mure og Volde mod Angreb udenfra ; men de havde ikke tcrnkt paa den Pest og Hunger, der raadede indenfor Murene . Bekjendelsesskrifterne vare <b>offentlige</b> Troens Vidnesbyrd for Ven og Fiende ; men de vare ikke ufeilbare og ingen Troens Regel , der kunde scettes ved Siden af Skriften . Ligesaa fri stod han ligeoverfor Luther . Han kjendte Luthers Skrifter nsie , han satte dem meget hsit ; imidlertid var der dog ogsaa i dem Feil og Skrsbeligheder som i ethvert menne skeligt Vcrrk . — Spener var overmaade mild ligeoverfor andre Konfessioner . Guds Naaderige strakte fig vidt ud over den lutherske Kirke , omendskjsnt det stod fast , at intet andet Kirkesamfund kunde sammenlignes med den , hvad angik Lcerens nsie Overensstemmelse med Guds Ord . — Ligeoverfor Katholi kerne var han dog meget kraftia . i fine Udtalelser ; thi han fandt hos dem , hvad han scerlig havde meget imod : menneskeligt Magtsprog i Troessager , den vilkaarligste Forvanskning af de kristelige Sandheder og Gjerningshellighed i Livet . Han saa i det pavelige Rom Johs . Aabenbarings  Babel . " Hvad Paulus engang sagde om sia . selv , nemlig at han havde dagligt Overlsb og Bekymring for alle Memgheder , det kunde ogsaa passe paa Spener ; ogsaa om ham kunde det hede :  Hvem er skrsbelig , nden at ogsaa jeg er skrsbelig ? Hvem bliver forarget , uden at det brcrnder mig ? " Dertil kom

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008071112028'target=_>Jouy, Victor Joseph Étienne de, 1833, Moralen anvendt paa Politiken</a><h4>1894</h4>Overalt , hvor Menneskene have et <b>Fcedreland</b> og Love , have de <b>offentlige</b> Dyder frembragt prii våte Dyder . Den modige og opossrende Borger har der den simke og trofaste <b>Qvinde</b> til Ledsagen inde ; den samme Fslelse besjeler dem : de ville Alt , hvad der kan ha ? dre Foedrelandet , de afskye Alt , hvad der tan bestjcemme dette . Saalcrnge Nom var frit , saaloenge de Overvundnes Skatte blot be ; rigede det <b>offentlige</b> Skatkammer , var <b>Scrdelighed</b> ligesaa almindelig blandt Qoinderne , som Tapt perheo blandt Moendene . Friheden var allerede gaaen tilgrunde under nogte adelige Familiers forl deroelige Rigoom og 3 Ergjerrighed ; men endnu vare Saederne rene : Julius , der , i det han fon stjod Pompeja , sagde , at Ccesars 3 Egtef < rlle ikke engang maatte vcrre mistlrnkt , bragle de romerste Qvinders forsvindende Kydsthed den siosie Hylding . Men Octavius og hans Efterfsli Zere , som frygteoe Frihedens Tilbagevenden , tilveier bragle en uoverstigelig Hindring derveo , at de new reve alle de af del gamle Roms Republicaners

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014102408022'target=_>Leonhardi, Gustav, 1876, Nat og Morgen</a><h4>1635</h4>indfandt sig foran hans Huus og bleve bespiste af ham . Naar en Fremmed spurgte efter hans Bolig , pleiede man at sige : „ Der , hvor du finder en Mcengde <b>Fattige</b> forsamlet, der boer Eligius . " — Han loskjobte de Slaver , som <b>offentlig</b> bleve frembudte til Salg . Mangen Gang erholdt indtil halvtredsindstyve Mcend og <b>Kvinder</b> paa eengang Friheden af ham . De Loskjobte lod han vwlge , om de vilde drage hjem , eller om de vilde blive hos ham og lade sig undervise i Chrisiendommen . Ikke sjelden havde han den Glwde hist og her at kunne opelste en vakker Plante for Himmerigets Rige . Saaledes blev en af de sacysiste Slaver , som han underviste i Christendommen, og som hed Theau , senere en Pryd for Christi Kirke , og Eligius fandt lidt efter lidt Anledning til at bescette hele sit Huus med reddede Mennesker eller til at gjsre dem dygtige til Kirkens Tjeneste . Senere byggede Eligius i Egnen ved Limoges et Kloster , hvortil Kongen havde stjcenket ham Grunden . Da man var fcerdig med Opbyggelsen af Klosteret , opdagede Eligius til sin store Forskrækkelse , at Bygningsfolkene havde gjort Bygningen en Fod bredere end den stjcenkede Grund og saaledes havde indtaget lidt mere Jord , end Kongen havde givet ham . Grcedende kom han til Kongen , bekjendte dette Bedrageri og bad om Tilgivelse . Kongen blev inderlig bevcrget over en saadan Troskab i det Smaa , stjcenkede ti ! Kirke og Kloster det Dobbelte af den for anviste Grund og sagde til sine Hoffolk : „ Se , hvilke trofaste og samvittighedsfulde Folk de ere , som tjene Christum . Mine Embedsmcend gjsre sig ingen Samvittighed af at stjcele hele Stykker <b>Land</b> fra mig , medens Eligius sittrer ved Tanken om , at han har tilegnet sig en Fodbred <b>Land</b> . " I Aaret 641 udvalgt til Biskop af Noyon , virkede denne fortræffelige Mand endnu med stor Trostab for Evangeliet , saavel blandt Christenmenighederne som blandt Hedningerne , og gik den Iste December 6 s9 , ledsaget afSines Velsignelser og Bonner , ind til sin Herres Glcede .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014082608009'target=_>Leonhardi, Gustav, 1862, Fortællinger fra Missionen i den gamle Kirkes tid</a><h4>1643</h4>indfandt sig foran hans Hnus og bleve bespiste af ham . Naar en Fremmed spurgte efter hans Bolig , pleiede man at sige : „ Der , hvor du finder en Mcengde <b>Fattige</b> for ? samlet , der boer Eligius . " Han lsskjobte de Slaver , som <b>offentlig</b> bleve frembudte til Salg . Mangen Gang erholdt indtil halvtredsindstyve Mcend og <b>Kvinder</b> paa eengang Friheden af ham . De Lsskjobte lod han vElge , om de vilde drage hjem , eller om de vilde blive hos ham og lade sig undervise i Chrisiendommen . Ikke sjelden havde han den Glssde hist og her at tunne opelste en vakker Plante for Himmerigets Rige . Saaledes blev en af de sachsiste Slaver , som han underviste i Christendommen, og som hed Theau , senere en Pryd for Christi Kirke , og Eligius fandt lidt efter lidt Anledning til at descette hele sit Huus med reddede Mennesker eller til at gjore dem dygtige til Kirkens Tjeneste . Senere byggede Eligius i Egnen ved Limoges et Kloster , hvortil Kongen havde stjcenket ham Grunden . Da man var fcrrdig med Opbyggelsen af Klosteret , opdagede Eligius til sin store Forskrækkelse , at Bygningsfolkene havde gjort Bygningen en Fod bredere , end den sijcrnkede Grund og saaledes havde indtaget lidt mere Jord , end Kongen havde givet ham . Grcrdende kom han til Kongen , bekjendte dette Bedrageri og bad om tilgivelse . Kongen blev inderlig bevceget over en saadan Trostab i det Smaa , stjcrnkede tii Kirke og Kloster det Dobbelte af den fsr anviste Grund og sagde til sine Hoffolk : „ Se , hvilke trofaste og samvittighedsfulde Folk de ere , som tjene Christum . Mine Embedsmand gjsre sig ingen Samvittighed af at sijcele hele Stykker <b>Land</b> fra mig , medens Eligius sittrer ved Tanten om , at han har tilegnet sig en Fodbred <b>Land</b> . " I Aaret 641 udvalgt ttl Bistop af Noyon , virkede denne fortræffelige Mand endnu med stor Troskab for Evangeliet , saavel blandt Chnsienmenighederne som blandt Hedningerne , og gik den Iste December 659 , ledsaget af Sines Velsignelser og Bsnner , ind til sin Herres GlEde .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111903041'target=_>, 1858, hygieiniske Congres i Kjöbenhavn, Juli 1858</a><h4>607</h4>Befolkningen paa Osterfælled maatte være en heel anden end den , der fandtes i Arbeiderboligerne i Christiania ; dette grundede han paa det betydelige Leie-Tab , man havde lidt og endnu bestandig led ude paa Osterfælled , hvortil kom , at en ikke ganske ringe Deel af Befolkningen der ved Vintertid modtog Understottelse af den <b>offentlige</b> eller private Velgjorenhed , hvorved den altsaa stilledes , i Klasse med de egentlige <b>Fattige</b> , hvilket jo ikke kunde være Tilfældet med Befolkningen i Arbeiderboligerne i Christiania . Foretagendet her var jo ogsaa nyt , og man havde derfor ikke kunnet gaae ind paa nogen Sondring af den Befolkning , der tyede til Bestyrelsen . Kunde imidlertid Foretagendet betydeligt udvides , troede Taleren , at man kunde komme til at bygge bedre i mange Retninger for den bedre stillede Klasse af Befolkningen , og derfor ansaae han det for et Gode , at man vedblev efterhaanden ' at faae nye Bygninger , fordi man efterhaanden vilde faae dem bedre i mange Henseender og tillige faae en bedre Deel af Befolkningen , der kunde grupperes efter Leiens Storrelse , som nu her var overmaade lav . Taleren havde forovrigt overbeviist sig om , at disse Boliger havde ovet en meget ' god Virkning saavel i physisk Henseende — i det han havde seet mange Born komme derud som vantrevne og skrophulose , der vare voxede op til sunde Born — som i <b>moralsk</b> Henseende . For en væsentlig Deel troede han , at dette beroede paa Bygningernes Beliggenhcd udenfor Byen , og deri laae Svaret paa Prof . Hornemanns Sporgsmaal , om man skulde bygge i eller udenfor Byen . Han ansaae det som en nodvendig Betingelse ved et saadant Foretagende for den Deel af Befolkningen , man her nærmest tænkte paa , at bygge udenfor Byen , og Grunden dertil laae i , at man maatte tåge Naturen til Hjælp . Osterfælled var saaledes for Beboerne i Arbeiderboligerne derude deres . Dagligstue , deres Bornekammer , og deri laae Kilden til den Sundhed , som for en stor Deel fulgte med disse Boliger . Han var overbeviist om , at inde i Kjobenhavn vilde disse Boliger , selv om de havde mere Luft end de havde , langtfra i physisk Henseende have bragt saadanne Resultater ; den <b>qvindelige</b> og Barnebefolkningen levede næsten ligesaameget paa Fælleden som inde i Husene , og deri laae Kilden ikke alene til deres Sundhed , man ogsaa til deres <b>moralske</b> Forbedring ; i sidste Henseende maatte han fornemmelig meget insistere paa det Moment , at man havde givet enhver Beboer en Have paa 6 Qvadratalen ; thi det var efter hans fulde Overbeviisning den , som havde forbedret mangfoldige Familieforhold . Naar Manden om Aftenen kom hjem fra sit Arbeide, ligesom ogsaa om Sondagen , gik han og syslede i sin Have , og det var en Fornoielse at see det ene Rækværk fremstaae smukkere end det andet , hvilke de med egne Hænder bragte istand ; i deres smaae Haver pleiede de ikke alene Blomster , men ogsaa Kjokkensager til Vinterbrug . Taleren havde seet mangen Mand , paa Veien til Drikfældighed, blive en ganske anden Mand , efterat han derude havde lært at sætte Priis paa den Hygge , som en god og sund Bolig gav , og paa den Adspredelse , som fulgte med at dyrke disse smaae Haver . Han maatte derfor holde paa , at disse Boliger bleve saa frit liggende som muligt udenfor Byen ; thi man vilde aldrig kunne opnaae lignende Resultater som der , ved at kjobe Boliger i tæt bepakkede Qvarterer af Byen , for at omdanne dem til Arbeiderboliger .

<h4>1328</h4>Professor Le vy maatte indromme , at den af Cancelliraad Sager an forte Maade , paa hvilken man gik frem i Berlin , ganske vist angav den Vei , man her nærmest burde tænke at gaae , naar man vilde virke for spæde Pleieborn . Den svage Side ved Institutionen hos os var nemlig , som tidligere bemærket , Valget af Pleieforældrene og Tilsynet med dem , i hvilken Henseende der var meget at onske anderledes . Baandetmellem Moder og Barn havde man principmæssigt sogt at vedligeholde , idet man lod Moderen selv sorge for Barnets Pleie mod en ugentlig Pengehjælp dertil . Valget af Pleieforældrene var altsaa overladt Moderen , hvorfor det ofte faldt sletud ; og forst i de senere Aar havde man indfort en Kontrol , der tvang Moderen til at skifte Pleieforældre , naar Forholdene vare altfor slette . Stammen af de nuværende Pleieforældre horte imidlertid i Almindelighed til de fattigste og daarligste Klasser , og hvad der opnaaedes ved at tiltvinge sig andre Pleieforældre , skete altfor ofte paa Bekostning af Bomene , for hvem den hyppige Skiften af Pleie maatte ansees fordærvelig , saameget mere som mangen Gang Skiftningen ikke forte til bedre Pleie . Den bestaaende Kontrol af en Dameforening med en fast ansat Dame-Inspectrice , der samvittighedsfuldt rogtede sit Kald , var vistnok meget god , men mere af <b>moralsk</b> end af administrativ Betydning ; den var ikke Herre over Forholdene og kunde ikke virke med den Frihed og Sikkerhed , som for den administrative Kontrol var nodvendig . Taleren vilde derfor ansee det for rigtigst , at virke for den Forandring af Systemet , at Pleieforældrene bleve satte under <b>offentlig</b> Kontrol, saa at ingen Pleieforældre turde tåge Born i Pleie , med mindre de havde stillet Garanti gjennem Politiovrigheden for , at de horte til en saadan Klasse i <b>Samfundet</b> , at man kunde betroe dem Born , og at de ikke vare saa <b>fattige</b> , at de ikke kunde give Barnet den fornodne Pleie . Dertil fordredes imidlertid ganske vist en Forhoielse af Pleiehjælpen , altsaa pecuniaire Offre , hvortil Regjeringen just ikke hidtil havde viist synderlig Tilboielighed . Som Folge nemlig af de store Byrder , Forsorgelsen af Pleiebomene medforte for Statskassen , gjorde man , allerede ide forste Decennier af dette Aarhundrede , betydelige Indskrænkninger i Understottelsen til Pleiebornene , og medens senere Fordringerne til hvad , der skulde ydes , i mange Henseender vare stegne , ljavdeman jevnlig formindsket Pengeunderstottelsen . Medens saaledes Pengeunderstottelsen fra Regjeringens Side tidligere havde naaet 48,000 Rdlr . , udgjorde den i Lobet af Aarene 1816 — 24 kun 44,000 Rdlr . , men fra den Tid dalede den til 24,000 Rdlr . , hvilke man ide allerseneste Aar endog havde formindsket med 4,000 Rdlr . En saadan Formindskelse af Pengeunderstottelsen kunde kun skee paa Pleiebornenes Bekostning , og viide man derfor virke alvorligt til Bedste for disse , maatte det skee Haand i Haand med Regjeringerne og Communerne , som ikke blot maatte sorge for et virksomt administrativt Tilsyn , men ogsaa maatte yde tilstrækkelige Pengemidler dertil , i den Overbeviisning, at de hertil medgaaende Udgifter kun skete i Statsformaalets og Almenvellets velforstaaede Interesse .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010011103018'target=_>Hansen, Maurits, 1841, Almindelig Verdenshistorie fra de ældste indtil vore Tider</a><h4>298</h4>maatte havs fyldt det tredsindstyvende Aar . Disse giords Übkast til Lovene og forelagde dem for Folkeforsamlingen , der oftest bestod af Spartas , sjeldnere ogsaa af det ovrige Lakonienslndbyggere — De overste Statsmcend vare de fem Eforer . Disse udvalgtes kun paa eet Aar ; de havde Opsigt med Lovenes Overholdelse og selv med Kongcrnes Forhold . Dette var Grundlaget for den spartanske Forfatning . Men herved blev Lykurg ikke staaende ; han vilde fremkalde en Almeenaand hos <b>Nationen</b> , for hvilken al Enkeltmands Interesse skulde trcrde tilbage . For Spartas Mre og Uafhcengighed skulde enhver Borger vcere beredt til i egentligste Forstand at opoffre Gods og Liv . Egennytte , Rigdom , Overdaad og Forsinelse af Scederne skulde forebygges . Af disse Grundscrtninger fremgik enhver Indretning . Der skulde hverken sindes rige eller <b>fattige</b> Borgere . Til den Ende bleve Landeiendomme fordeelte lige imellem alle Familier , og hverken ved Kjob , eller Arv maatte Nogen tilegne sig Mere end den ham tilkommende Lod . Al Handel med Udlcendinger var forbuden , og derfor ogsaa Brugen af Guldog Solvmynt . For at overholde den eensformige , simple Levemaade forordnede Lykurg , at alle Borgere skulde spise i Fcellesskab . Svagelige Born bleve strax udsatte i en Skov og overladte til Bytte for Mangel eller vilde Dyr . Legemsovelser , som heller ikke <b>Pigerne</b> fritogcs fra , udgjorde den vigtigste Deel af Opdragelsen , hvilken var <b>offentlig</b> og under strceng Opsigt . Man voennede Drengene til Haardfsrbed / til at udholde legemlige Smerter , ste Dsden med Foragt imode . Den stolteste Patriotisme blev dem indprcentet ; de loerte Lovene udenad og maatte give korte og bestemte Svar paa ethvert Sporgsmaal . lovrigt bekymrede man sig lidet om deres Aandsudvikling . Vaabensuelser , Brydning , Veddelob , lagt vare Mcendenes eneste Beskjeftigelse , thiAgerbrug ogHaandvcerkeroverlodes de ulykkelige Heloter , hvis haarde Behandling ikke engang Egennytten kunde formilde , eftersom de vare hele Statens , ikke den enkelte Herres Eiendom . Naar man frygtede for det tiltagende Antal af Heloterne , tillodes det endog de unge Spartanere som en Vaabenovelse at overfalde og drcebe dem .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112301026'target=_>Munch, P.A., 1876, Samlede Afhandlinger</a><h4>1502</h4>Folkets Tidsfordriv og Forlystelser i Rom og de omliggende Egne har jeg allerede berørt , forsaavidt jeg har omtalt Tombola ' en og Carnevalet . Tombola ' en afgiver , som man vel maa antage , Hoved- og Glands-Punktet i de <b>offentlige</b> Forlystelser , og der er ingen , hvori Menigmand Bvneß at tåge ivrigere Deel . De forekomme heller ikke saa sjeldent , idet de sedvanligviis foranstaltes af det <b>Offentlige</b> ved de større Kirkefester, i de mindre Steder navnligen paa Kirkemesse- og Skytshelgen-Dagene, hvor ogsaa Hesteløb pleje at holdes og Festen at slutte med Fyrverkeri . Men naar man undtager disse Lejligheder, hvor , merkeligt nok , de religiøse Fester højtideligholdes med saadan Morskab , som man hos os vilde finde uforenelig dermed og alene henlegge til de politiske , skeer der egentlig lidet for den almindelige Fornøjelse , og Folket selv synes , trods Bin Lyst og gode Tid til at more sig , at savne Opfindsomhed eller Drift til at kaa no ^ en ret Morskab istand . Vist er det , at den , der kommer til Rom med Forestillinger om et bevæget <b>offentligt</b> Liv , om glimrende Folkefester , om Sang og Klang og Dands , vil føle sig meget skuffet ; han vil med Forundring erkjende , at der f . Ex . i og ved IHsdenliavn er ueudeligt mere Lystighed og Folkemorskab end i Rom og de Smaabyer , der omgive den . Maaskee er dette at søge i det Tryk , som et foreldet og mistænkeligt Regjerings system legger paa Folket , maaskee og i altfor stor og hemmende Indgriben fra Gejstlighedens Side , i den Hensigt at overholde <b>Sedeligheden</b> . Saavidt jeg har erfaret , gjelder dette idetmindste hvad and sen angaar. De fleste af mine Læsere skulde dog vel mene , at dersom der nogensteds dandses med Liv og Sjæl , saa maatte det være i Italien , ved hvis blotte Navn der vekkes Forestillinger om Saltarello og Tarantella og hvad nu alle de andre nationale og ikke nationale Dandse hede . Men herved maa jeg nu først og fremst gjøre den Bemerkning , at Italien aldrig har været et Dandsens <b>Land</b> . De gamle Romere dandsede ikke , og der er ikke videre Spor af , at deres Efterkommere gjøre det mere . Man er altfor tilbøjelig til i dette Henseende at forvexle Spanien med Italien . I Spanien dandser man ; der er Dandsen virkelig national , og , hvad der maa til , naar den skal have nogen Betydning som Folkefornøjelse , den dandses af Mænd og <b>Kvinder</b> tilsammen . I Italien derimod er den saakaldte nationale Dands kun en Skygge af , kvad den i de fleste andre Lande plejer at være , fordi den savner hiin Hovedbetingelse , at dandses

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008052703001'target=_>Scott, Walter, 1837, Bedstefaders Fortællinger, eller Den nyere Skotlands Historie</a><h4>152</h4>Jacob oergrede sig over sine hungrige Landsmcrnds Graadighed og Paalroengknhed , og forargede sig des , Uden over det <b>fattige</b> og usle Udseende , som mange af dem havde ved Siden af de rige Engelffmcend , hvilket bragde Vanoere over det <b>Land</b> , hvortil han selv , ligesaavelsom de , henhsrte . Han sendte derfor Ordre til Regjeringsraadet i Skotland , at de stulde forbyde saadanne paatrcengende Menneffer at forlade deres Fsdeland , i det han klagede over deres <b>Scrder</b> » g Udseende , som noget der maatte bringe Vancere over alle de Indfsdte i Skotland . En Prollamation blev fslgeli ^ t udstscdt i Edingburg . hvori Regjeringen besv ^ rede sig over , at et si ^ Antal Mcrnd og <b>Qvinder</b> af ringe Stand og Vilkaar , uden Ncrrings , vei eller noget at leve af , begave sig ° fra Skotland til Hoffet , somvarncestm opfpldt med dem , til stor Plage for hans Majestoet og til betydelig Vancere sor den stotste <b>Nation</b> ; thi da disse Beilere til Hoffets Gunst , efter alle deres Omdsmme . som saae dem , vare orkes , lose Skelmer eller usle , <b>fattige</b> Stakler , saa opvakte deres Paatrcengenhed og Mangde den Mening > ar


<h2>“Kvindens funksjon i den private sfære” [Woman’s function in the private sphere]</h2><h3Kvinde_, Privat_, Sædelighed_, Land_, De uprivilegerte _</h3><h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008082512001'target=_>Luther, Martin, 1863, Dr. Martin Luthers Udførlige Udlæggelse af første Mosebog</a><h4>191</h4>holder sig i Gudsfrygt og beder , end at man stnlde cengste og plage sig med Frygt for de tilkommende Ting . Men derom en anden Gang . 127. Saa er det nu min Mening , at man ilke saa sikkert kan grunde de astrologiske Forudsigelser paa denne Text ; thi det er Tegn , som Fornuften iagttager og samler . Men den ncermeste og egentlige Mening er denne , at vi forståa , at Moses taler om de Tegn , som Gud viser os , foråt Mennestene i Almindelighed derved skulle enten formanes eller forfcerdes . Og dette stal vone nok om den fjerde Dag . 128. Men her antydes for os Sjcelens Udodelighed , eftersom ingen Skabmng uden Mennestet alene kan forståa Himlens Lob eller maale og begribe Himmellegemerne . Et Svin , en Ko eller en Hund kan Ne gjore sig noget Begreb eller nogen Tanke om det Vand , som de drikke ; men Mennesket har Begreb og Skjon paa Himlen og alle Himmellegemer . 129. Derfor skinner her en Gnist as det evige Liv , at Mennestet af Naturen maa ove sig i saadanne Tings Betragtning og Erkjendelse . Og denne Omsorg viser , at Mennestene Me ere skaute til , ultid at leve paa Verdens laveste Sted , men at de tilsidst skulle jaa Himlen i Eie , hvilken de i dette Liv betragte med Forundring og ove sig i Betragtning og Studermg paa himmelske Ting . 139. Og hvis Ne det var saa , hvortil havde det da tjent , eller hvorfor havde det ogsaa vceret fornodent , at Gud saa rigelig havde ladet Mennestet vide saadanne Ting , hvem dets eget Legemes Skikkelse og Retning give Vidnesbyrd om , at det horer til de himmelske Ting ; hvorvel dets Begyndelse er same svag , <b>fattig</b> og elendig . Thi det forste Menneske blev gjort af en lordklump ; derefter begyndte den menneskelige Slcegt at avles og formeres ved Mandens og <b>Kvindens</b> <b>Sced</b> . Deraf bliver efterhvert Fosteret i Moderlivet uddannet med alle Lemmer og voxer , indtil det ved Fodselen kommer til Dagen og dette Himlens Lys . Derncest begynder det forst at fole , arbeide og rore sig . 131. Men nåar ester alt dette Legemet er kommer til sin Kraft og Styrke , og i et sundt Legeme Fornuften og Forstanden er i Virksomhed , da viser sig forst det forstandige Liv , som Ne er i andre jordiske Skabninger, saaledes at Mennesket ved de mathematiste Videnstabers Uedvirken og Hjcelp , om hvilke Ingen kan sige Andet , end at Gud har aabenbaret og vist dem , fra Jorden styver med sit Sind og sin Tanke til det Hoie , lader de jordiske Ting fare og udforsker og betragter de himmelske . Dette gjore ilke Kjor , Svin eller andre ufornuftige Dyr ; det gjor Mennestet alene . Derfor er Mennesket en Skabmng , som er stabt til endelig at forlade , Jorden og i Himlen leve i en evig Tilvcerelse . Thi deraf kommer det , at han Ne alene kan tale og domme ( hvilket henhorer til Dialektik og Rhetorik ) , men ogfaa kan lcere den hele Lcere om Himlens Lob , Stjernernes Bevcegelfe og hvad den menneskelige Fornuft mere kan begribe af disse hoie og hemmelige Ting . 132. Derfor aabenbares nu fra denne tredie Dag af vor Wre , at nemlig Gud nu tcenker og foretager sig at stabe en saadan Skabmng , som kan erkjende og forståa Bevcegelsen af disse Himmellegemer , der ogsaa stabes paa den fjerde Dag , og fom kan have Glcede af en saadan Erkjendelse , hvortil ogsaa dens Natur er scerdeles tilboielig , og som passer sig for den . Men alt dette skal bevcege og opvcekke os til Tcitsigelse , at vi som Vorgere henhore til dette <b>Fcedreland</b> , som vi nu se paa med Forundring og forståa , men endnu kun som Fremmede og Landstygtige ;

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009091403021'target=_>Luther, Martin, 1864, Dr. Martin Luthers Udførlige Udlæggelse af første Mosebog</a><h4>1019</h4>med de ydre Sandser , og maa lcere at leve og fore sin Vandel efter de Ting , forn man ikke ser , og som man ikke kan gribe med de ydre Sandser . Det er Kjodets rette Dodelse , hvilket altid er kjodelig sinder og vil vcere ganske og aldeles trygt , vil kun holde sig til de Ting , som ere ncervcerende og synlige ; foler det derfor det Modsatte , faa bliver det angst og bange . 266. Derfor stal man foreholde Folk dette Exempel , hvorledes Jakob indscettes til Konge og Prcest og indvies til Regjeringen og Prcestedommet ; thi dette er sandelig en sorgelig Stads og en ynkelig Maade at salve og indscette en Konge paa . Man iforer ham ingen festlig Dragt , man fmykker han : ikke med nogen kongelig Hat eller Krone , man giver ham intet i Scepter Haanden ; men man forsyner ham med en Pose og en Stav , og dermed jages han saaledes ud i Landflygtighed ; men Velsignelsen forbliver hos hans Broder Efau , hvem den dog ingenlunde tilkom . Men tilslut , da Jakob saaledes er bleven dodet ved Troen paa den usynlige Gud , folger ogfaa det fynlige Gode og besidder hans <b>Sced</b> det forjcettede <b>Land</b> , og af hans <b>Sced</b> fodes Christus , den evige Konge og Prcest , hvis Rige og Prcestedomme ogsaa indeholdes i denne Velsignelse . 267. Saaledes salvedes ogsaa David til Konge over Israel , da Savl var bleven afsat og forkastet af Gnd , 1 Sam . 16 , 1. 13. ff . Men desuagtet forbliver dog Savl i Regjeringen forn for ; han har Held i sine Foretagender hjemme i Freden og ude imod Fienden ; ja , han forfolger desuden gruelig David forn en Ovrorer , der med List hemmelig efterstrcebte hans Kongevcerdighed , og David maa imidlertid vanke omkring , som En , der er flygtig og elendig i det <b>Land</b> , hvor han var indsat og erklceret for Konge af Gud . Historien vidner , at han i hele ti Aar saaledes maatte flakke omkring , medens Savl daglig forfulgte ham og hemmelig efterstrcebte ham . Synes dn , det er et Kongerige, nåar En i sit eget <b>Land</b> maa vcere fremmed og elendig i hele ti Aar ? Hvad kunde vel vcere underligere og übilligere , end at En skal udncevnes og bestikkes til Konge uden Nige , uden Scepter , uden vis Bolig , ja saa , at han ikke alene beroves <b>Land</b> og Rige , men ogsaa sit eget <b>Hjem</b> , <b>Hustru</b> og Born ? 268. Men Christi Exempel overgaaer alt dette . Han var foragtet , og vi llgtede ham intet , siger Propheten Es . 53 , 3. Da han vil fare til Himlen og indgaa til sin Herlighed , da han vil overvinde Doden , Synden og Djcevelen , nagles han til Korset ; han doer og begraves som den mest Foragtede blandt alle Mennester og Djcevle . Er det at indgaa til Herlighed eller overvinde Doden og triumvhere over den ? Ja , det er at indgaa til Herlighed . Thi det er vor Herre Guds hemmelige og forborgne Veie , som man ikke maa bedomme efter Kjodet og den menneskelige Forstand , men efter Skriften , og saaledes som Christus selv forstaaer og tyder dem , nåar han siger til sine Disciple Lue . 24 , 26. : burde det Me Christum at lide alt dette og at indgaa til sin Herlighed ? 269. « Netop paa samme Maade maatte ogsaa Jakob i tyve Aar vcere i Elendighed , og saa forst blive Konge og Prcest . David maatte stakke hid og did i ti Aar og soge mangen en Krog , hvor han kunde ligge sikker een Nat , undertiden vel kun een Time , er det en Konge , af Gud indsat og bckneflet i Riget ? Ja , saaledes horer du ham synge og hoit rose sig af sin Kongevcerdighed ' Ps . 63 , 12. : Kongen stal

<h4>3468</h4>49. Men i 5 Moseb . , hvor der Kap . 25 , V . 5. befales , at den ene Broder stal ovvekke den anden <b>Sed</b> , indefluttes ogsaa Polygamiet , det er , at man kunde have mere end een <b>Hustru</b> ; dette bekreftes ogfaa nf Moses . Dette var ogsaa Grunden til , at Salomo havde saa mange Hustruer , af hvilke Nogle vistnok ogfaa vare beflegtede med ham centen ved Blodsforvandtstab eller Svogerstab , og var der en <b>fattig</b> <b>Kvinde</b> i <b>Familien</b> , har hun vel holdt sig til Kongen som sin Frende eller Svoger . Han har vel erneret mange af dem , og dog havde han uden Tvivl ogsaa mange Medhustruer foruden Dronningen , i Kraft af denne Lov , hvilket sandelig var en trykkende og utactlelig Ting , nemlig at vere besveret med saamange Koner eller Medhustruer , til hvem man dog hverken havde Lyst eller Kjerlighed .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014082608023'target=_>James, John Angell, 1872, unge Kvinde</a><h4>72</h4>hans skjenne Karakter . Ham bM de altid betragte ikke blot som deres Sjceles Frelser , men ogsaa som deres Rettigheders Forsvarer , deres Freds Beskytter . Kristendommens fuldkomne Afskaffelse af Flergiftet har medftrt en stor Forbedring i <b>Kvindens</b> Stilling . Hvor Flergiftet hersker , maa Kvindekønnet altid vcere i en Tilstand af dybt Forfald og Elendighed . Erfaringen aflcegger mange og sørgelige Vidnesbyrd om , at Flergiftet fornedrer og flover baade Legeme og Sjcel og gjFr <b>Samfundet</b> fremmed for disse cedle og milde Indvirkninger , hvilke tilbørlig beskyttede vilde befindes at vcere arvelige endog for vor faldne Natur . Findes der vel et eneste Erempel paa , at Flergiftet ikke har vceret Kilden til mange og bitre Ulykker faavel i det huslige Liv som i Staten ? Har det nogensinde opdraget en dydig og for Himmelriget stikket Slcegt ? Har det nogensinde udmcerket sig ved <b>moralske</b> Dyder ? Eller bar det ikke meget mere udmcerket sig ved den forfiVrdeligstc TMeslMhed ? Ved dets Udøvelse , som har hersket saa vidt i ncesten alle Lande og alle Tidsaldre , hvor Kristendommen er har vceret kjendt eller herskende , taber Mgteflabet al surhed, al Renhed og gjensidig Tillid . Den cegtestabelige Lykke ombyttes med Lysternes Tilfredsstillelse : <b>Kvinden</b> er blot til for at Pirre Mandens Begærlighed istedet for at bidrage til hans hnslige Hygge , og <b>Hjemmet</b> forvandles ncesten til et SlMehus . Der kuune vcere mange Elskerinder , blot en <b>Hustru</b> ; der findes maaste mange Mlid ' re , men ingen Moder- Mhed ; der opftres kun Scener af uendelig Strid , vcekt ' et og u « ret af Uviud og Skinsyge , for hvilke den huslige Hygge altid maa fly . Der findes intet , fom opmuntrer til Forbedring , ingen Anledning til Troflab , intet Anste om at behage ; thi hvorledes skulde saadanne Melser kunne nceres af en <b>Hustru</b> , der hvilkensomhelst Dag kan fortrcenges af en ny Favorit ? Under saadanne Omstændigheder findes hverken Plads eller Anledning til Udviklingen af de Dyder , der udgøre Kvindekønnets Hceder . Det er just Kristendommens cerefulde Fortjeneste at have gjenopreist og gjenindsat Mgtcstabet i dets oprindelige Renhed , at have gjengivet <b>Kvinden</b> hendes Lykke , hendes Personlighed , hendes Stilling i Fannlien,

<h4>173</h4>Kristendommens , Merlandenes eller Vesterlandenes , Fortidens eller Nutidens , og vi skulle finde , at intet saa voesentlig adskiller dem som <b>Kvindens</b> Tilstand . Enhver <b>Kvinde</b> , hun vcere nu gift eller ugift , rig eller <b>fattig</b> , har dog en Kreds , indenfor hvilken hun udover en vis Magt , enten til Skade eller Gavn . Enhver <b>Kvinde</b> vil nødvendigvis ved sin Daarstab eller sin Klogskab , sin Dyd eller sine Laster forege sit Folks Vcerdighed eller paaskynde dets Forfald . Saalcengc <b>kvindelig</b> Dyd blomstrer , beskyttet af det ene og agtet af det andet KjM , kan et Folk ikke synke altfor dybt . Til en vis Grad er det <b>Kvinden</b> , som opretholder sit Folks Lykke . Hendes Dyd staar , hvis den er fast og ufordærvet, som en beskyttende Vagt over Folket . Lov , Retfærdighed og Frihed , Kunst og Videnskab , alt bidrager naturligvis til et Folks Velgaaende , men hvis <b>Kvindens</b> almindelige <b>Sædelighed</b> staar saa er alt andet sorgjceves , medens derimod , hvis <b>kvindelig</b> Dyd og Dannelse er almindelig, Civilisationens Stro ' m stiger til sin Drste og bliver frugtbringende for den mest udstrakte Overflade . Et <b>Samfund</b> , hvori <b>Kvinden</b> opfylder sit Kald , falder ikke lettelig ; thi den Magt , hendes cedlc Hjerte udMer paa andre Hjerter , vil lMe det af Elendigheden og gjengive det Glcede og Lykke . Se der , faavcl udenfor fom indenfor det ægteskabelige Liv hur <b>Kvinden</b> en Mission , og det ei en übetydelig . Hendes Virkekreds er Samsuudet , hendes Maal er dets Velbefindende , hendes BelMning dets Taknemlighed og Erkendtlighed .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014112108104'target=_>, 1871, Nordisk Conversationslexicon</a><h4>786</h4>i Slaget ved Lepanto 7 Oct . 1574 , hvor han foruden to andre Saar fik den venstre Haand og Arm lammede for bestandig . Helbredet i Messina gjorde han forst under Admiral Marco Antonio Colonna et Tog med til Levanten , der- Pllll et til Afrika under Don Juan d ' Austria . Ester endt Krigstjeneste gik han til Neapel , hvorfra han indskibede sig til Spanien , men i Sept . 1575 blev Skibet taget af Ssrsvere . 5 Aar tilbragte C . fom Fange i Algier ; forst havde han to grufomme Renegater , derpaa selve Deyen til Herre . Han viste sig mod sine Medfanger som en opoffrende og god Kammerat , og tog alene Skylden paa sig for mange dristige , men mislykkede Planer til Flugt , hvilket flere Gange ndfatte ham for Piinsel og Dsd . Med stor Anstrengelse fik hans uformuende <b>Familie</b> ham lsskjM 1580 ; <b>fattig</b> og forladt kom han til sit <b>Fcedreland</b> , maatte igjen tage Musketten og drog med paa et Tog til Portugal og Azorerne 1581 og 1582. I Portugal vandt han hvad der dengang var en Sjeldenhed hos en Spanier Kjcerlighed til Landet og dets Literatur , af hvilken sidste man fporer Indflydelse i hans tidligere Prosauarker . Kjed af Krigerlivet trak han sig tilbage til sin Fodeby , udgav 1584 sin Hyrderoman » 62 l2tea « , hvis Helt og <b>Heltinde</b> ere Forklædninger af ham og hans Elskede , med hvem han kort efter blev gift ; forresten er Bogen skreven i den courante ssdlig-fade Hyrdestiil . Kort efter forfattede han sin fsrste Roekke Dramaer ( 20 — 30 ) , der trods deres Übehjelpsomheder og mange allegoriske Figurer indeholde Scener af eiendommelig Kraft og Skjsnhed , ifcrr hvor han fcetter Begivenheder af sit eget bevcrgede Liv ind deri ; navnlig har » Numaucik « Partier , den stsrste Digter vcrrdige . Hans Haab om at ernoere sig ved Arbeide sor Scenen glippede , og c . 1588 drog han til Sevilla , hvor han fik en lille Bestilling som Opkroever af resterende Skatter , men det laae ham noermere , naar han paa Embedsvegne gjennemstreifede Andalusien , poetisk at nyde det rige Natur- og Folkeliv , end nsiagtig at udfore sine Opkrævninger ; han kom i Gjeld til Regjeringen for en übetydelig Sum og fad en Tid lang i Fcmgfel . <b>Fruentimmerne</b> i hans <b>Familie</b> ernoerede sig kummerlig ved Syning under hans bestandig trange Kaar . Forgjceves anssgte han Regjeringen om en Post i Amerika , fom ellers Vamfundets Udskud brugtes til . Fra Sevilla stal C . vcere dragen til la Mancha ( hvor han lader Don Quixote have sin Hjemstavn ) og der ssgt sit Erhverv som Skatteopkræver for Ordenen San Inan og ligeledes vcere bleven fcengslet paa Grund af Chicane fra Indbyggernes Side . 1603 finde vi ham bosat i Valladolid , hvor han atter kom i Famgsel tilligemed sin <b>Familie</b> , fordi dens Medlemmer havde varet Vidner til en Kjoerlighedsduel om Natten udenfor deres Bolig . I Valladolid har C . væsentlig udarbeidet Iste Deel af » von Quixote « , der udkom i Madrid 1605 ; og det fslgende Aar drog han til Madrid, hvor han gjorde Bekjendtstab med Literaturens Notabiliteter , men trods det store Bifald, fom hans Bog vandt , bleve hans Kaar ikke stort bedre . Don Quixote har gjort Epoche i Verdensliteraturen . C.s ncermeste Hensigt var , som bekjendt , at parodiere de phanwstiste og uri-

<h4>1829</h4>Corday d ' Armans ( Marie Charlotte ) , den svarmeriste , heltemodige unge <b>Pige</b> , der troede at redde Frankrig ved at dråbe Marat , var Datter af en <b>fattig</b> Adelsmand , s . 1769 i St . Sllturin i Narheden af Caen . Hun blev opdraget i et Kloster , og udmcertede sig blandt sine levnaldrende ved sin Forkærlighed for publicistist og historist Låsning , navnlig af Plutarchs Biografthier og Rousseaus Skrifter , ved sadelig Alvor og glsdende Begeistriug for alle ophsiede og cedle Formaal i Livet . Revolutionen gjorde derfor et starkt Indtryk Paa dette dybe og varme Gemyt ; jo mere hun årede de Mand , der havde kampet for Mennesterettighederne og for den sande Frihed , desto mere afskyede hun dem , fom ved Raahed og Vlodtorst vare lVkyld i Revolutionens Udsteielser og Radster . Efterhaanden modnedes saaledes i hendes Sjal det Forsat at bidrage til Fædrelandets Frelse ved at rydde en af de vcrrste Tyranner afveien . Et Numer af Marats clir hvori han yttrede , at der endnu maatte falde 200,000 Hoveder , bestemte hende til at gjsre ham til sit Offer . Hun reiste nnder et Paastnd fra sit <b>Hjem</b> , vaklede efter Ankomsten til Paris mellem Robes- Pierre og Marat , men bestemte sig omsider atter for denne . Efter fsrst om Morgenen at vare llfviist af hans <b>Huusholderste</b> kom hun dog samme Aften ind til ham , medens han var i Badet , og idet han opskrev Navnene paa nogle af hende ncevnte foregivne Sammensvorne med den Mtring: „ De stulle fane deres Lem ; jeg stal lade dem Alle guillotinere i Paris " , stsdte hun Uhyret en Kniv , som hun havde holdt skjult , i Brystet 13 Juli 1793. Hun blev strax greben ; paa Veien til Fangstet styrtede en begeistret Ingling frem og vilde dse for hende ; han blev myrdet af det rafende Folk . 17 Juli gjorde Guillotinen Ende paa den nnge Svarmerstes Liv ; hun dsde med Rolighed og vcrrdig Fasthed som en Martyr for Friheden . Boddelens Raahed foraarsagede et Udbrud af Uvillie hos Mamgden , og en Stemme raabte : „ See , hun er stsrre end Brutus!" men ogsaa den Mand , fra hvem dette Udraab kom , maatte dse . Sorbetter kalde Rebstagerne et Mellemprodukt, fom er staget af 3 Dukter og faaledes ligner almindeligt 3 staget Tougvark ( Trosser ) , men er bestemt til at slaaes endnu engaug 3 eller 4 sammen i modsat Retning for at give 3 - eller 4 staget Kabelgods . Cordeliers bencevnes i Frankrig de regelbundne Franciskanermunke efter det Reb ( emclo ) , de boere om Livet . C . kaldtes under den fsrste franske Revolution Medlemmerne af en 1790 stiftet politisk Klub i Paris , fordi de havde deres Forlamlingsted i en Klosterkirke , der tilhsrte C . Denne Klub indbefattede de vildeste Anarchister , som her uhindret prcedikede den raa Sansculottisme og Had mod al Religion , <b>Moral</b> , Orden og Monarchi . Med Jakobinerne laae de jevnlig i Strid , mindre dog om politiske Principer end om Anfsrernes Indflydelse og om sieblikkelige Partioiemed . Fra begge disse Revolutiousleire udgik alle de Folkevagelser og Udsteielser , som beherskede Begivenhedernes Gang og Nationalforsamlingen, og som medfsrte de blodige Voldsscener og Omvaltninger , der betegne Revolutionen. Danton , Marat , Fabre d ' Eglantine ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015010648109'target=_>, 1852, Almennyttigt dansk Konversations-Lexikon over alt det Videværdigste i Naturen, Kunsten og Videnskaben, fra den ældste til den nyeste Tid, i alfabetisk Orden, efter Brochhaus's Tydske Original, nyeste Udgave</a><h4>2543</h4>Danskes Omgoengelse mindst ; da derimod en Fransos droeber mig med Grimacer , en Tydst med vidtlsftig Snak , en Engellcender incommoderer mig med fin Selvros , en Spanier med fin Grauitet . " „ Men paa det at Ingen fial tcrnte mit Forsoet udi dette Kapitel er alene at rose dette Foll , maa jeg ogsaa af Fornedenhed vende det lidt staa den anden Side , og eftersee de Feil , som de Danfie tillcrggeS . Og sinder jeg da , at der ikke uden Grund tilloegges dem en vis Magelighed , saa at fornemme Folk doer af Sksrbug , efterdi de ille gider gaae fra et HuS til et andet , og gemene ( simple ) Folk doer af Armod , efterdi de ilke fiietter om Arbeide . " Endvidere anfsrer han LEresyge , Beundring for alt det , der kommer fra fremmede Lande , Begierlighed efter at vide noget af Alting , hvilken Begierlighed foraarsager endogsaa blandt de bedste Hoveder , at man neppe kommer til nogen mcerkelig Fuldkommenhed i nogen Ting . Som Erempler ncevner han Lysten til at loere sire til fem Sprog og til at lcere at behandle fiere musikalske Instrumenter . Ligeledes bebreider han Folket , „ at det har giort sig til Slave af visse unedvendige Moder , der betynger dem , der sidder i Husholdning faaledeS , at dette <b>Land</b> , besynderlig Kiebenyavn , er blevet en af de kostbareste Stoeder i Europa at leve udi . Thi , hvorvel Alting er her een Gang saa godt Kiob som i Paris , saa lever dog en borgerlig <b>Familie</b> med langt sterre Bekostning her end der . Der lader en hel <b>Familie</b> fig neie med to til tre Voerelser , her ikke uden med et helt HuS , der gaaeS tilfods , her ageS , der har man nok i een <b>Pige</b> , her beheves Stue-Pige , Kokke-Pige , Barne - <b>Pige</b> , Amme , foruden Lalei , Kudfi og GaardStarl og gemenlig Skolemester udi Huset , saasom det lader ilde at satte fornemme Borgerbern i publike Skoler ; der regnes Udgifter i Klcrder og Fede , her tilligemed udi Drille - Penge , Pige-Bryllupper , prcegtige Begravelser , Barselstuer, Nytaarsgaver o . s . v. Thi mangen Borger slipper ille med tresindstyve Rigsdaler aarlig i Nytaarsgave , mangen temmelig god Dedsbo bliver til intet , naar Bekostningerne paa Begravelsen afregnes , og mangen HaandvoerlSmand soetter til paa en eneste Bryllupsdag Alt , hvad han har lagt til gode udi fine UnglarlSdage . Man kunde vel svare hertil , at Landet itle liver derved , at Pengene roulerer blandt Folk , men de mange Domestiker , som underholdes i hvert HuS og tryller en Husfader af Alting allermest , har den Virkning , at hver HaandvoerkSmand maa forskrive fremmede Drenge og Svende , eftersom den ftsrfte Del af Landets <b>fattige</b> Ungdom lader sig enrvllere til Kudfie , Lakeier , GaardSlarle o . s . v. "

<h4>4420</h4>blev derncest Erkebisp i Smyrna og 1795 Patriarch i Conftantinopel . Allerede 1798 , da Franfimcrndene havde bemcegtiget sig Wgvpten og man beskyldte Groekerne for at staae i hemmelig Forbindelse med dem , fordrede den Tyrkiske Pobel PatriarchenS Hoved ; men Selim var overbeviist om HanS Uskyldighed , og G . formanede i flere Hyrdebreve Groekerne til ei at giore fcelles Sag med Franfimcrndene . Vel forviste man ham til AthoS , men kun for at beskytte ham mod Pobelen , og han gienindsattes snart i sin Vcerdighed . Da 1806 de Russiske VaabenS Held og en Engelsk FlaadeS An « komft for Conftantinopel paany ophidsede Tyrkerne mod Groekerne , hvorved PatriarchenS Liv tom i Fare , forvifte Selim ham atter til Bierget AthoS , men kaldte ham efter nogen TidS Forlob tilbage . G . vandt siden mere og mere almindelig Agtelse ved Idmyghed , Menneskekioerlighed og Velgisrenhed ; han levede tarveligt, holdt strengt over <b>Scedelighed</b> hos de Groefic Geistlige og anvendte sine Indkomster til fromme Diemed : til <b>Fattige</b> uden Hensyn til deres Tro , til Skoler , til Gicnoprettelsen af Bogtrykkeriet i Conftantinopel og til nyttige Skrifters Trykning . Han befordrede Anlæggelsen af Skoler med indbyrdes Undervisning i SkioS , PathmoS , Smyrna , Athen , Sparta og Kandia . HanS Prædikener vidner ligesaa meget om Fromhed og Taalsomhed som om Mennefiekundfiab . Men da GrcekerneS Opstand paa Morea udbrod 1821 , blev ogsaa han Porten fordcegtig . . For at afvcerge den allerede besluttede Myrden af Grcrkerne i Conftantinopel udtalte han vel den 21 de Marts 1821 den af Divanet paa en truende Mande forlangte Bansoettelse mod IpsilantiS , Souzo og alle Deltagere i Opstanden , samt udstedte ligeledes til Geistligheden et Hyrdebrev , som giorde Grockernc streng Lydighed mod Porten til Pligt ; dog hans Undergang var allerede besluttet . Da Fyrst MurusiS ' S <b>Familie</b> , som efter HanS Henrettelse var overgivet til G . ' S Opsigt , ved den RuSsifie Gesandts Bistand hemmelig var bragt ombord paa et Skib , der fsrte dem til Odessa , blev G . ( uagtet Alt var fieet uden HanS Vidende og han strar havde giort Anmeldelse derom , saasnart han erfarede Flugten ) ferfte Paafledag , d . 22 de April 1821 , efter HoimessenS Afholdelse ved sin Udgang af Basilikaen uden videre paa Storherrens Befaling grebet af lanitscharer og tilligemed tre Bifiopper og otte Geistlige i sit Ornat hcengt foran Kirkens Hovedport . Paa hans Bryst befoeftede man DodSdommen , som lsd paa : „ at han havde vceret vidende om sine LcmdSmcendS Opstand paa Morea og hsist sandsynlig virret Sammensvcergelsens hemmelige Hovedmand , hvorfor den bele Grcrfie <b>Nation</b> , mgtet . den

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009091503033'target=_>Ahlfeld, Friedrich, 1866, Menneskelivet</a><h4>404</h4>Barmhjertige , treenige Gud , gjsr os til sande Tjenere af din Majestet og A3re . At tjene Dig er den hsieste Frihed og Salighed . O , hvi stulde vi ikke gjsre det ! Tjener Du os ikke hver Dag ! Sidde vi ikke til Bords , medens Du bcerer frem for os , hvad der er fornodent for Livet ! Har din kjcere Slm ikke paataget sig en Tjeners Skikkelse og hengivet sig for at tjene enhver stakkels Synder og gjsr det endnu i den Helligaand med al sin Kjccrlighed og sine hyihellige Naadegaver ! — Saa giv os da ogsaa Tros7ab og ?)d----myghed til at tjene hverandre med den Gave , vi have faaet som gode Huusholdere over Guds mangeslags Naade ( 1 Ped . 4 , 10 ) . Forbarm Dig iscer over alle dem , der som Tjenestefolk trccde ind i deres Ncestes Huus . Lad dem tjene for dit Aasyn . Lad dem fremfor Alt bekjende :  See , jeg er Herrens Tjener , Herrens <b>Tjenerinde</b> " . Giv dem 3 ) dmyghed , Sanddrnhed og Troskab , at de ikke ssge deres Eget , men vandre Dig til Behag og af al Evne fremme Husets Bedste . Lad dem hjerteligt slutte sig til <b>Familien</b> , lad dem h « Zre med til den lille Menighed i <b>Hjemmet</b> . Giv dem alle Naade , at de af deres Herrer kunne modtage en Velsignelse for det evige Liv , men at de ogsaa igjen tunne vcere til Velsignelse for Herrerne . Det gjere Du for Jesu Christi Skyld . Amen . Uden gjensidig broderlig Tjeneste er Livet utcenkeligt og umuligt . Christi Kirke er det store Legeme , og vi ere Alle Lemmer derpaa , som skulle tjene hverandre hver med sin Gave . Den Kloge skal tjene sine Brsdre med sit Raad , den Stcerke med sin Styrke , den Rige med sin Nigdom , og den <b>Fattige</b> tjener atter den Nige med sine Krcefter . Men Alle skulle tjene hverandre med den Kjcerlighed , der er fsdt af Herren . — Fra den celdste Tid af har der altid vceret Folk , som lcvede i deres Ncestes Huus og vare til Tjeneste og Hjcelv for dem . Elieser tjente Abraham , Jakob tjente Laban , losva tjente Moses . Men forudeu faadanne frie Tjenere gaves der ogsaa tjenende Folk af et ringere Slags , nemlig Slaver og Slavinder , og om dem taler Paulus i vor Tert . Saadanne Tjenestefolk have vi , Gud vcere lovet , ikke mere i vort <b>Fcedreland</b> ; Ordet Slave har nu en skrekkelig Klang for os . Men det er ikke blot denne haarde og ufrivillige Tjeneste , som man er bange for , man vil nu allerhelst slet ikke mere tjene ; thi Egenvillien og Hovmodet ere blevne saare mcegtige blandt

<h4>621</h4>Som enhver Stand saaledes har ogsaa den ugifte Stand sine Farer . Vi ville her slet ikke tale om dem , som forblive ugifte for at redde Kjodets Frihed og deres tilvante Magelighed . Men vi ville iscer minde om , at de Ugifte ofte vedblive , endogsaa naar de ere oppe i Aarene , at tjene alslags Forfcengelighed og Tant . Andre scette sig mere og mere fast i deres Egenvillie og Egensindighet » ; deres syndige Scerheder blive ikke modarbeidede ved den daglige Opoffrelse for Mand eller <b>Hustru</b> , men blive bestandig skarpere og mere udproegede . Have de intet bestemt Kald , saa udfylde de ofte Tiden med Synder i Ord og tilbringe deres Liv med alskens Snak . Hertil komme Klager over Eensomhed og over enhver lille Modgang ; de mene sig uretfcerdig behandlede af Gud ; de betragtc med Vitterhed deres Sodfkende , som leve i et lykkeligt Familieliv ; de ere skarpe i deres Domme og blive til Vyrde baade for sig selv og for Andre . Hele deres Liv dreier sig om dem selv . At saadanne Mennesker kun lidet svare til det Billede af de Ugifte , som Apostelen Paulus udkaster , behover ikke at omtales . — Hvorledes kan der da bsdes paa alt dette ? Hvorledes kunne de holde sig frie for saadanne Forvildelser ? Vi give tre Svar herpaa :  Ved at leve i et inderl igt <b>Samfund</b> med Herren , ved at ssge sig en bestemt Virkfomhed og ved at slutte sig til en <b>Familie</b> . " Den daglige Omgang med Herren bevarer ogsaa den Eensommes Hjerte aabeut , frisk , glad , deeltagende og tilgcengeligt . Den modarbeider Egenvillieu , Egensindigheden og Forstenelsen i egne Tanker , Anskuelser og Luner . Den bevarer Kjcrrligheden , saa at man kan bede for Andre , arbeide og opoffre sig for dem . Vi have kjendt eenlige Mennesker , som i den stille Omgang med den dyrebare Sjceleven havde vundet en Rigdom af Kjcerlighed , Venlighed , Idmyghed og Taalmodighed , som man ikke skulde have troet at finde i deres eensomme Kammer . Jesus Christus formaaer jo overalt at udrydde vort Hjertes Forkeerthed , Trods , Haardhed og Bitterhed og at gyde himmelsk Veulighed og opoffrende Kjcerlighed i det . Den , der omgaaes med ham , har den rette Ven . , Broder , Mand og Swttestav . — — Vi komme nn til at omtale den bestemte Virksomhed og Familielivet . Den katholske Kirke har et Tilflugtssted for de Ugifte i Klostrene , og den evangeliske Kirke har et stort Antal Stiftelser for adelige og borgerlige Jomfruer . Begge Slags Anstalter kunne til en vis Grad erstatte Familielivet , men blive dog kun til velsignede Herberger for den Eensomme , naar et eller andet Kald er forbundet med Opholdet i dem . Hverken ved Von eller fromme Betragtuinger eller Andagtsovelser eller ved Arbeider til Nytte for sig selv formaaer man at udfylde Livet . Der maa tomme en Virksomhed til , som gaacr ud paa at vise Andre hjelpsom Kjcerlighed , og som tillige kan vcere en Beskyttelse mod mange indvortes og udvortes Fristelser . Derfor have ogsaa de Klostre og Stiftelser , som beskjceftige sig med at pleie Syge og opdrage rige eller <b>fattige</b> Born , altid vceret de meest velsignede for deres Medlemmer eller Beboere . I de andre har det trods alle Regler og

<h4>824</h4>Saaledes ere de Christne Kongeborn , der vandre som Fremmede paa Jorden , men have hjemme i Hunlen ; der leve i Foragt og Ninghed , men dog ved deres Tro og Bsn stulle beherske Verden . Det er Guds Bsrns kongelige Stand . Salig den , som lever i den sikkre Vevidsthed om denne Herlighed og Mre , og som aldrig slipper den . Om hvilket Menneske og hvilken <b>Familie</b> vilde vi nu christeligt talt sige , at det er gaaet tilbage med dem ? Ikke om dem , som ere blevne <b>fattige</b> . Noomi vender hjem fra Moabiternes <b>Land</b> med sin Svigerdatter Nuth . Hendes Eiendom har Dyrtiden fortceret , hendes Mand og hendes Ssnner hvile i det fremmede <b>Land</b> ; hun er faaledes hjemssgt , at hun ikke mere vil hedde Noomi , den V ) ndefulde , men Mara , den Bedrovede . Og dog er det ikke gaaet tilbage med hende ; thi hun har bevaret sin Gudsfrygt og sit Haab paa Herrens Naade . — Den gamle Tobias lever i Ildlcendighed med sin <b>Familie</b> . Hans Formue er tabt , et tyrannisk Aag hviler Paa hans og hans Folks Nakke , hans Mnes Lys er udslukket , og hans <b>Hustru</b> ernccrer ham ved at spinde . Og dog kan han sige til sin Son :  Frygt ikke , mit Barn , for vi ere blevne <b>fattige</b> ; Du haver Meget , om Du frygter Gud og viger fra al Synd og gjor det , som er behageligt for ham . " Udvortes taget er det gaaet tilbage med disse Mennesker , men indvortes staaer den gamle Tobias hsit over det fordervede Hedningefolk, blandt hvilket han lever forn Fange . — Joseph og Maria ere ifolge deres Herkomst cegte Kongebarn , LEtlinger af den beromteste Slcegt i Verden . Men Joseph er Tommermand i det ringe og foragtede Nazareth , og i Vethlehem kan Parret kun fiude Herberge i

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111903047'target=_>, 1838, Samling af Norge vedkommende Kgl. Rescripter, Resolutioner og Collegialbreve m.v. for Tidsrummet fra 1660-1813</a><h4>4695</h4>? 39. Alle Born i Staden , som ikke have Foreldre eller Formyndere, der kunne , eller andre , der ville berale for deres Skulegang , bor gives fri Underviisning , og Fattigvesenets Direction bur drage Omsorg for og medvirke til , at de nytte samme . F 40. Saasnart Directionen for Fattigvesenet er traadt sammen , setter den sig i Correspondence med Directiunen over det Kongsbergffe Skolefond , samt vedkommende Kongelige Departements , for at faae saa mange af de Skoler Fondet hidtil har vedligeholdt, som behoves til de <b>fattige</b> Vorns Underviisning , sig ooerdragne tilligemed de fornodne Indtcrgler t , l at bestride Udgivterne ved Underviisningen, samt til at bevirke andre Forandringer ved Skolevesenet , som maatte ansees nodvendige eller gaunlige for Fattigvesenets Oiemeed . F 41. Skolegangens Oiemced bor vere : at Bomene ved deres Sjels ug Legems Krefters rigtige Udvlkling og Anvendelse kunne dannes til oplyste , gode , arbeidsomme Mennester og Burgere ; at de til den Ende tldligen hindres fra Lediggang og Laster , og vennes til <b>Sedelighet</b> ) , Orden og Ncenligbed ; at de anfores til saadan Kundstab , sum er noduendig og gavnlig fur Mennesket og Borgeren i Almindelighed , og at oe undervises og oves i saadant Arbeide , hvorved de i Fremtiden , enten i Mangel afandet Neringsar » beide , kunde fortjene deres Underholdning , eller nyttigen beskjeftige sig i ledige Timer . F 42. Underviisningen i Skolerne stal altsaa vere deelt imellem V.denstab , Haandarbeide og nyttige Legems - Ovelser . F 43. Ved Videnstab forstaaes ikke her det , sum henhurer til blut Lerdum , eller blot speculativ Kundstab i egentlig Forstand , men det , som for den menige Almue er furnodent , fatteligt ug brugbarr , saasum : Lesning , Skrivning , Regning , Religion og Mural , almindelig Kundstab om Naiuren , dens Indremlnger og Virkninger , om borgerlige Indretninger og Pligter , om de vigtigste Sundhedsreglcr , om <b>Fedrelandets</b> Forfatning og Beliggenhet ) vg deslige , alt efter korte og simple Læreboger , som dertil kunne velges , og med Hensyn til den Forstjel , sum bor gjures i Underviisning for Drenge vg <b>Piger</b> . Ogsaa sorges sor , at Bomene anfores til rigtig ug harmonist Sang , iser Koraler og gode Folkesange- F 44. Haandardeider bor vere af det Slags , som i det daglige Liv kan sur Almuens Fult vere al » mindeligt og gavnligt , og sum for Bomene efter Furstjellighed af Kjon , Alder og Krefter findes passende , samt hvortil i Skulebygninaeme kan vere Leilighed og Rum , saasom : at pille Vomuld , at karde Uld og Bom » uld , at plukke og spinde Uld , Bomuld , Hor , Hamp og Blaar , at tvinde , haspe , vinde , spole , at binde og strikke , at kniple , at lappe paa Kleder , at stoppe Strumper , at knytte Fistegarn , at veve Bendler og uldne Baand , at gjore Hegler og Fistekroge , stette Kurve og Hatte , at forfcrrdige allehaande Legetoi og andet Numberger-Kram , og andet mere ; og for <b>Pigerne</b> tillige at tildanne og spe Linned og Kleder , m . v. Ved Arbeidets afvexlende Afdeling sees blandt andet ogsaa derpaa , at Legemskrefterne styrkes tilligemed oves . F 45. Til at give Underviisning i Haandarbeide ansettes ved hver af Fattigvesenets Skoler en Lcrremoder ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009072703011'target=_>Ahlfeld, Friedrich, 1863, Menneskelivet</a><h4>662</h4>vaager over din Herres Born med et christeligt Aie , at Du itte tier til deres Synder ; nåar han seer , at Du , som maastee om Dagen har luget Utrudt og saaet Koru , igjen om Aftenen kan luge Utrudt paa andre Steder og faae en anden Slags <b>Sed</b> . Pigen i Huset hos Syreren Naamau s 2 Kg . 5 ) bragte Troen paa den leveute Gud med til Syrien , og hun blev sin Herres Veileder til den ene og sande , naaderige Gud . Hun gav Naamcm , hvad hans Konge Benhadad ille kunde give ham , og Gud saae det og strev det iud i sin Bog . — Troe og ydmyge Tjenere ere meget verd i Guds Ame . De Gamle fortelle om en Eneboer , der tillagde sig felv en hoi Grad af Hellighet ) . Da lundgjorde Gud ham i et Syn , at han neppe havde brngt det fna vidt i Gudfrygtighed forn en <b>Pige</b> , der tjente i en vis Kro . Eneboeren begav sig paa Veien for at opsoge hende og fporge hende ud om hendes Vandel . Hun fvcirede ham , at hun itte var sig nogen seregen Hellighet , bevidst ; hun forrettede flittigt sit Arbeide iHuset og havde tillige den Vane , nåar hun bar en Dragt Brende op til Kjottenet , at tenke med hjertelig Kjerlighed paa ham , der havde baaret Korsets svere Tre af Kjerligled til os . — Men hvilken er den sidste og holeste Trost for Enhver , der tjener ? At Gud har bygget en Stad , hvor der hverken er <b>Fattige</b> eller Rige eller Trelle eller Frie mere ; hvor Alle ere frie i Christo Jesu . Herre Jesus , velsign Herrerne og Tjenerne , <b>Huusmodrene</b> og <b>Pigerne</b> , at de i Troen maae indgaae til Guds Borns herlige Frihed . Amen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112101017'target=_>Ahlfeld, Friedrich, 1866, Menneskelivet i Guds Ords Lys</a><h4>593</h4>alle arme Syndere , ligesom Han endnu stedse tjener os i den Hellig-Aand med al sin Kjcerlighed og alle sine hpihellige Naadegaver . Saa giv da ogsaa os Tr.oskab og Idmyghed til som gode Huusholdere over Guds mangeslags Naade at tjene hverandre med den Naadegave , Enhver as os har faaet . Men forbarm Dig iscerdeleshed over alle dem , der indtrcede som Tjenestetyende , Dreng eller <b>Pige</b> , i sin Ncestes Huus . Lad dem tjene som for dit Ansigt . Lad dem ftrst og fremst bekjende :  Se , jeg er Herrens Tjener , Herrens <b>Tjenerinde</b> . " Giv dem Wmyghed , Sanddruhed og Troskab , saa de ikke spge sit Eget , men omgaaes til dit fulde Velbehag og as yderste Evne ftge at fremme sit Herskabs Bedste . Lad dem med sine Hjerter slutte sig til <b>Familien</b> ; lad ogsaa dem hore til dens lille Menighed . Giv dem Alle Naade til as sit Herskab at annamme nogen Velsignelse for det evige Liv og til ligeledes vaa sin Side at kunne vcere sit Herskab til nogen Velsignelse . Det ville Du gjsre for Jesu Christi Skyld ! Amen . Uden gjensidig broderlig Tjeneste er Livet ikke tcenkeligt eller muligt . Christi Kirke er det store Legeme , og vaa det ere vi Alle Lemmer . Det ene Lem maa tjene det andet med sin Naadegave . Den Kloge maa tjene sine Brødre med sine Raad , den Stcerke den Skwbelige med sin Styrke , den Rige den <b>Fattige</b> med sin Rigdom . Og omvendt tjener den <b>Fattige</b> den Rige igjen med sine Krcefter . Men Alle skulle tjene hverandre med den as Herren fMe Kjcerlighed . Lige fra den graae Oldtid har der bestandig vceret Mennesker , som have givet sig md i Andres Huse for at tjene og hjcelpe dem . Eliefer tjener Abraham ; Jakob tjener Laban ; losva tjener Moses . Ved Siden as saadanne frie Tjenere gaves der ogsaa Tjenere og Tjenesteviger i Ordets haardefte Betydning , nemlig Slaver eller Trcelle , Slavinder eller Trcelkvinder . Til disse var det Paulus talte i vor Text . Saadanne ftnoes gudskelov ikke lcenger i vort <b>Land</b> . Deres blotte Navn indjager Skrcek . Dog er det ikke alene denne haarde , ufrivillige Tjeneste , man frygter for ; men i vor Tid saae Enhver helst , at han ganske kunde undgaae at tjene . Selvraadigheden og Hovmodet ere blevne mcegtige blandt Menneskene . lordbrugere og Driftsherrer, der have tungt Arbeide , klage over at de ikke lengere kunne faae Tjenestefolk . Alle hige ester lettere og i deres egne Tanker sordeelagtigere Beskjeftigelser . Enhver vil allerede i sin Ungdom vcere fri og selvstendig . Unge Karle og <b>Piger</b> ville heller arbeide i Fabrikker end vcere i Tjeneste . O , de vide endnu ikke , hvilket langt storre Vcerd det huuslige Liv har end Livet blandt den store fra- og tilstrpmmende Mcengde , der bringer Synder fra alle Verdens

<h4>1098</h4>Vistnok maa ogsaa mangen <b>Familie</b> beljende :  Vinteren er for os den « lykkeligste Tid , " Noget , som forpvrigt er dens egen Skyld . Hvor nemlig ikke Guds Fred boer i Huset , der hersker der mere Kiv og Trcette om Vinteren end om Sommeren . Om Sommeren har man lettere for at gaae afveien for hinanden . Hertil kommer desuden , at trange Kaar fples mere trykkende om Vinteren . Som en Fslge af at man ikke bcerer sin Nsd i Herren skyder den Ene Skylden paa den Anden . Man kives om Brsd , om Klceder , om Lys , om Varme og andre Ting . Men i Grunden kives man , sordi man ikke ftnder nogen Smag i Livsens Vrpd , og fordi det er saa nwrkt og koldt i Ens Indre . En Vinter uden Herren er et Helvede i Huset ; en Vinter med Ham er en dyrebar Naadestid og en Haandpenge paa den salige Hvile i Gud . Og som for det huuslige Liv , saaledes har den ogsaa sin Betydning for Folke- og Menighedslivet .  Fred paa Jorden ! " sang Englene ved Bethlehem . I een Forstand tiltvinger Gud sig nceften med Magt en overfladisk Fred i Norden . Krigene maae nemlig da tie , og imidlertid have de stridende Parter Tid til at tcenke over sin Synd og Skyld , over sine udDvede Rcedselsgjerninger og over Freden . Gud driver Menneskene til Fred , men endnu mere til Barmhjertighed. Det kan ikke negtes , at Npden hviler tungere paa de <b>Fattige</b> om Vinteren end om Sommeren . Da har den christelige Kjcerlighed den meest udstrakte Mark for sin Virksomhed. Da byr den ikke glemme at gjsre vel og at meddele; da bjsr den bespge Faderlsse og Enker i deres Trcengsel, aabne sit Huus for de Elendige , klcede de Nsgne og prcedike Evangelium for de <b>Fattige</b> . Naar Alt ligger stivnet og dodt , naar Marken hverken har Grces eller Blomster , da bpr den fra Eden overvlantede Blomst : den af Herren fsdte Kjcerlighed , iscerdeleshed blomstre . Kjcerlighed og Barmhjertighet ere Vedbenden og Sneklokken i de Christnes Have . Da er det Vinter og dog ikke Vinter ; da er det koldt og dog ikke koldt . Paa den Maade vil Gud , der ogsaa har skabt Vinteren , bruge den , som al Trcengsel , til sin 2 Ere og sin Menigheds Gavn og Bedfte . Nu vel , kjcere Lcesere , priser da Herren for alle Hans Gjerninger og Indretninger ! priser Ham , fordi Han itke lader Sommer og Vinter , Frost og Hede , <b>Sced</b> og Hsst , Dag og Nat ophpre ! priser Ham under Korset og i gode Dage ! men benytter ogsaa Trcengselen, de morke Dage og lange Vinteraftener ret alvorlig ! til Eders Frelse og til Guds Forherligelse i Menigheden ! Lader ikke Lcengselen efter det <b>Land</b> uddye , hvor der ikke lcenger er nogen Storm i Sjelen , og hvor de unge Skud paa Livstrceet ikke mere bortfryse og henvisne ! Herre vor

<h4>1184</h4>Saaledes ere de Christne Kongebarn , der vandre paa Jorden, men leve i Himmelen , der ere Fremmede og Udlcendinger herneden , men med sit Hjerte og Haab ere i Hjemmet, der maae dKe Foragt og Fornedrelse as Menneskene , men dog med sin Tro og Bpn opretholde og regjere Verden. Dette er Guds Bsrns kongelige Stand . Salig er den , som lever i en sikker Forvisning om og daglig Uopvelse as denne Herlighed og HKHed . Om hvilke Mennesker og hvilke Familier sige vi nu som Christne , at de ere gaaeoe tilbage ? Ikke om de forarmede. Noomi kommer tilligemed sin Svigerdatter Ruth tilbage fra Moabiternes <b>Land</b> . Hendes Gods har Dyrtiden berDvet hende ; sin Mand og sine Smner har hun begravet i det fremmede <b>Land</b> ; hun er bleven saa haardt hjemsogt , at hun ikke lcenger vil kaldes Noomi , Dnde , men Mara , Bitterhed . Og dog er hun endnu ovenpaa ; thi hun har endnu ikke sluppet sin Gudsfrygt og sit Haab til Herren . Den gamle Tobias er en landflygtig Mand tilligemed sin <b>Familie</b> . Han har mistet sit <b>Fcedreland</b> og sit Arvegods ; raat Voldsherredømme lcegger sin lernhaand paa hans og hans Folks Nakke ; hans Oines Lys er udslukket , og hans <b>Kone</b> erncerer ham ved at spinde . Ikkedestomindre siger han til sin Son :  Frygt ikke , mit Barn , fordi vi ere blevne <b>fattige</b> ; du har Meget , om du frygter Gud og viger fra al Synd og gjpr det , som er behageligt for Ham . " Udvortes ere djsse Mennesker komne paa Knceerne ; men indvortes ftaaer den Gamle hsit over det vanartige Hedning efolk , blandt hvilket han lever som Fange . Josef og Maria ere efter sin Herkomst cegte Kongebarn , fodte af den fornemste Slcegt i Verden . Men Josef er dog ikke Andet end en Tpmmermand i den lille foragtede By Nazareth , og i Bethlehem kan dette Par kun finde Herberge i en Stald . Om Maria og hendes nyfMe Barn siger en gammel Sanger :

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014011608145'target=_>James, John Angell, 1868, unge Qvinde</a><h4>119</h4>blot som deres Vjceles Frelser , men ogsaa som deres Rettigheders Forsvarer , deres Freds Beskytter . Christendommcns fuldkomne Afskaffelse af Flergiftet har medftrt en stor Forbedring i Qvindcns Stilling . Hvor Flergiftet hersker , maa QvindekjMnet altid vcere i en Tilstand af dybt Forfald og Elendighed . Erfaringen aftcegger mange og sørgelige Vidnesbyrd om , at Flergiftet fornedrer og slaver baade Legeme og Sjcel og <b>Samfundet</b> fremmed for disfe cedle og milde Indvirkninger , hvilte tilbørlig beskyttede vilde befindes at vcrre arvelige endog for vor faldne Natur . Findes der vel et eneste Erempel paa , at Flergiftet ikke har voeret Kilden til mange og bittre Ulykker saavel i det huslige Liv som i Staten ? Har det nogensinde opdraget en dydig og sor Himmelriget stikket Slcegt ? Har det nogensinde udmcerket sig ved <b>moralske</b> Dyder ? Eller har det ikke meget mere udmoerket sig ved den forfærdeligste TMcslMhed ? Ved dets Udsvclsc , som har hersket saa vidt i ncesten alle Lande og alle Tidsaldre , hvor Christendommen ei har voeret kjendt eller herskende , taber Mgte ' skabet al Omhed, al Renhed og gjensidig Tillid . Den ægteskabelige Vvlle ombyttes med Lysternes Tilfredsstillelse : <b>Qvinden</b> er blot til for at pirre Mandens Vegjcerltgbed istedet for at bidrage til hans huslige Hygge , og <b>Hjemmet</b> forvandles ncesten til et Sygehus . Der kunne vcrre mange Elskerinder , blot en <b>Hustru</b> ; der findes maaske mange Mpdre , men ingen Modcr- Mhed ; der opftres tun Scener af uendelig Strid , vcekket og nceret af Avind og Skinsyge , for hvilke den huolige Hygge alcid maa fly . Der findes Intet , fom opmuntrer til Forbedring , ingen Anledning til Troskab , intet Anske om at behage ; thi hvorledes skulde saadanne Folclser kunne nceres af en <b>Hustru</b> , der hviltensomhclst Dag kan fortrcenges af en ny Favorit ? Under saadanne Omstændigheder findes hverken Plads eller Anledning til Udviklingen as de Dyder , der udgjM QvindckjjZ ' nnets Hcrder . Det er just Christcndommens cerefulde Fortjeneste al have gjenoprcist og gjenindsat Wgtestabet i dets oprindelige Renhed , at have gjengivet <b>Qvinden</b> hendes Lykke , hendes Personlighed , hendes Stilling i Familien, hvilket Alt hun har vceret berøvet ved Flergiftet , og

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012052204016'target=_>Ahlfeld, Friedrich, 1869, Menneskelivet i Guds Ords Lys</a><h4>1507</h4>Vistnok maa ogsaa mangen <b>Familie</b> bekjende : „ Vinteren er for os den ulykkeligste Tid " , Noget , som forovrigt er dens egen Skyld . Hvor nemlig ilke Guds Fred boer i Huset , der herster der mere Kiv og Trcette om Vinteren end om Sommeren . Om Sommeren har man lettere for at gaae afveien for hinanden . Hertil kommer desuden , at trcmge Kaar foles mere trykkende om Vinteren . Som en Folge af at man Me bcerer sin Nod i Herren styver den Ene Skylden paa den Anden . Man kives om Vrod , om Klceder , om Lys , om Varme og andre Ting . Men i Grunden kives man , fordi man Me sinder nogen Smag i Livsens Brod , og fordi det er saa morkt og koldt i Ens Indre . En Vinter uden Herren er et Helvede i Huset ; en Vinter med Ham er en dyrebar Nciadestid og en Haandpenge paa den salige Hvile i Gud . Og som for det huuslige Liv , saalededes har den ogsaa sin Betydning for Folke- og Menighedsttvet . „ Fred paa Jorden ! " fang Englene ved Bethlehem . I een Forstand tiltvinger Gud sig ncesten med Magt en overfladist Fred i Norden . Krigens maae nemlig da tie , og imidlertid have de stridende Parter Tid til at tcenke over sin Synd og Skyld , over sine udovede Rcedselsgjerninger og over Freden . Gud driver Mennesket til Fred , men endnu mere til Barmhjertighed. Det kan ilke negtes , at Noden hviler tungere paa de <b>Fattige</b> om Vinteren end om Sommeren . Da har den christelige Kjcerlighed den meest udstratte Mark for sinVirtsomhed. Da bor den Me glemme at gjore vel og at meddele; da bor den besoge Faderlose og Enker i deres Tromgsel, aabne sit Huus for de Elendige , klcede de Nogne og prcedike Evangelium for de <b>Fattige</b> . Naar Alt ligger stivnet og dodt , nåar Marken hverken har Grces eller Blomster , da bor den fra Eden overplantede Blomst : den af Herren fodte Kjcerlighed , iscerdeleshed blomstre . Kjcerlighed og Barmhjertighed ere Vedbenden og Sneklokken i de Christnes Have Da er det Vinter og dog Me Vinter ; da er det koldt og dog Me koldt . Paa den Maade vil Gud , der ogsaa har stabt Vinteren , bruge den , som alTrcengsel , til sin ZSre og sin Menigheds Gavn og Bedste . Nn vel , kjcere Lcesere , priser da Herren for alle Hans Gjerninger og Indretninger ! prifer Ham , fordi Han Me lader Sommer og Vinter , Frost og Hede , <b>Sced</b> og Host , Dag og Nat ophore ! priser Ham under Korset og i gode Dage ! men benytter ogsaa Trcengselen, de morke Dage og lange Vinteraftener ret alvorlig : til Eders Frelse og til Guds Forherligelse i Menigheden ! Lader Me Lcengselen ester det <b>Land</b> uddoe , hvor der ilke Iceuger er nogen Storm i Sjelen , og hvor de unge Skud paa Livstrceet Me mere bortfryse og henvisne ! Herre vor

<h4>1599</h4>Saaledes ere de Christne Kongeborn , der vandre paa Jorden, men leve i Himmelen , der ere Fremmede og Udlcenoinger hernede , men med sit Hjerte og Haab ere i Hjemmet, der maae doie Foragt og Fornedrelse as Mennestene , men dog med sin Tro og Bon opretholde og regjere Verden. Dette er Guds Borns kongelige Stand . Salig er den , som lever i en sikker Forvisning om og daglig Udovelse as denne Herlighed og Hoihed . Om hvilke Mennester og hvilke Familier sige vi nu som Christne , at de ere gaaede tilbage ? Ikke om de forarmede. Naomi kommer tilligemed sin Svigerdatter Ruth tilbage fra Moabiternes <b>Land</b> . Hendes Gods har Dyrtiden berovet hende ; sin Mand og sine Sonner har hun begravet i det fremmede <b>Land</b> ; hun er bleuen saa haardt hjemsogt , at hun ikte lcenger vil kaldes Noomi , Inde , men Mara , Bitterhed . Og dog er hun endnu ovenpaa ; thi hun har endnu ikke sluppet sin Gudsfrygt og sit Haab til Herren . Den gamle Tobias er en landflygtig Mand tilligemed sin <b>Familie</b> . Han har mistet sit <b>Fcedreland</b> og sit Arvegods ; raat Voldsherredømme lcegger sin lernhaand paa hans og hans Folks Nakke ; hans Ames Lys er udstukket , og hans <b>Kone</b> erncerer ham ved at spinde . Ikkedestomindre siger han til sin Son : „ Frygt ikke , mit Barn , fordi vi ere blevne <b>fattige</b> ; du har meg et , om du frygter Gud og viger fra al Synd og gjor det , fom er behageligt for Ham . " Udvortes ere disfe Mennester komne paa Knceerne ; men indvortes staaer den Gamle hoit over det vanartige Hedningefolk , blandt hvilket han lever fom Fange . Josef og Maria ere ester sin Herkomst cegte Kongeborn , fodte af den fornemste Slcegt i Verden . Men Josef er dog ikke Andet end en Tommermand i den lille foragtede By Nazareth , og i Bethlehem kan dette Par lun finde Herberge i en Stuld . Om Maria og hendes nyfodte Barn siger en gammel Sanger :

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014121208040'target=_>Ahlfeld, Friedrich, 1878, Menneskelivet i Guds Ords Lys</a><h4>643</h4>alle arme Syndere , ligesom Han endnu stedse tjener os i den Hellig-Aand med al sin Kærlighed og alle sine hoihellige Naadegaver . Saa giv da ogsaa os Troflab og ' ? ) d myghed til som gode Huusholdere over Guds mangeflags Naade at tjene hverandre med den Naadegave , Enhver af os har faaet . Men forbarm Dig iscerdeleshed over alle dem . der indtrcede som Tjenestetyende , Dreng etter <b>Pige</b> , i sin Na-ftes Huns . Lad dem tjene som for dit Ansigt . Lad dem forst og fremst bekjende : » Se , jeg er Herrens " Tjener , Herrens <b>Tjenerinde</b> . " Giv dem Ydmyghed , Sanddruhed og Troflab , saa de ikke soge sit Eget , men omgaaes til dit fulde Velbehag og af yderste Evne foge at fremme sit Herskabs Bedste . Lad dem med sine Hjerter slutte sig til <b>Familien</b> ; lad ogsaa dem hore til dens lille Menighed . Giv dem Alle Naade til af sit Herskab at annamme nogen Velsignelse for det evige Liv og til ligeledes vaa sin Side at kunne vcrre sit Herskab til nogen Velsignelse . Det ville Du gjsre for Jesu Cbristi Skyld ! Amen . Uden gjensidig broderlig Tjeneste er Livet ikke tcenkeligt eller muligt . Christi Kirke er det store Legeme , og vaa dtt ere vi Alle Lemmer . Det ene Lem maa tjene det andet med nn Naadegave . Den Kloge maa tjene sine Brodre med sine Naad , den Stcerke den Skrøbelige med fin Styrke , den Rige den <b>Fattige</b> med sin Rigdom . Og omvendt tjener den <b>Fattige</b> den Rige igjen med sine Krcrfter . Men Alle skulle tjene hverandre med den af Herren fsdte Kjcerlighed . Lige fra den graae Oldtid har der bestandig vcrret Mennesker , som have givet sig ind i Andres Kuse for at tjene og hjcrlpe dem Elisser tjener Abraham : Jakob tjener Laban ; losva tjener Moses . Ved Siden af saadanne frie Tjenere gaves der ogsaa genere og Tjenestepiger i Ordets haardefte Betydning , nemlig Slaver eller Trcrlle , Slavinder eller Trælkvinder . Til disse var det Paulus talte i vor Text . Saadanne findes gudskelov Me lamger i vort <b>Land</b> . Deres blotte Navn injager Skra-k Dog er det ikte alene denne haarde , ufrivillige Tjeneste , man Nvgter sor ; men i vor Tid saae Enhver helst , at han ganske kunde undgaae at tjene . Selvraadigheden og Hovmodet ere blevne mcegtigs blandt Menneskene . Jordbrugere og Driftsherrer, der have tungt Arbeide , klage over at de lamaer » kunne saae Tjenestefolk . Alle hige efter lettere og i egne hanker fordelagtigere Vesljceftigels er . Enhver vil allerede ism Ungdom vcere fri og selvstændig . Unge Karle og Piaer ville heller arbeide i Fabrikker end vcere i Tjeneste . O , de vide endnu ikke hmlket langt stsrre Vcerd det hunslige Liv har end Livet blandt den store fra- og tilstrsmmende Mcenqde , der bringer Synder fra alle Verdens Kanter sammen under eet

<h4>793</h4>Ved Bryllupsmaaltidet maa Bordbonnen ikke mangle . Har den end lcengs sovet i en <b>Familie</b> , saa er dog dette en Dag paa hvilken den bor vcekkes . Og har man aldrig bedet i en <b>Familie</b> , saa maa man dog paa en saadan Dag fole . hvor <b>fattig</b> , hvor tom , hvor kold man scetter sig tilbords , naar man ikke har takket Herren og ikke anraabt Ham om bans Naade i Fremtiden . Endnu vaaligger det efter almmdelia Vedtcegt Brudens Fader at forrette Bordbonnen paa denne Dag . Hans Vorn og Gjcester spise endnu ved hans Bord . Forsommes Vonnen paa denne Dag , Me <b>Pigens</b> Liv dermed i Forældrenes Huus , saa vil vel heller ikke Livet i hendes eget Huus blive begyndt dermed . Man undlader da at medgive Bornens en af de kosteligste Bygninasstene i Familielivet . Ved Vryllupsmaaltidet er det en stor Glcede at see begge det unge Pars Familier sammensmeltede til Eet i Herren . I Midten sidder det unge Wgtevar, og til Hsire og Venstre Fcedrene og Modrene . O , hvor ncer ere ikke disse nu bragte hinanden ! Deres Kolighed og Vonner hvile jo nu paa Bornenes fcelles Huus . Og efterdi Mand og <b>Kone</b> ere Eet , bor det unge Par ogsaa staae i et sandt Varneforhold til hinandens Forceldre . Gid enbver ung Mand og ung <b>Kone</b> ret af Hiertet vilde kalde sin Wgtewlles Forceldre Fader og Moder ! Gid Forceldrene ogsaa vilde betragte sit nye Barn med ret faderlige og moderlige Folelser ! Det gjor en <b>Hustru</b> ondt , naar hun vel har vundet sin Mands , men ikke tillige hans Forceldres Lnerte Da blceser det hende saa koldt unode : hanv <b>Familie</b> : hun forekommer sig selv som en Kvist , der er bleven rigtig indpodet . Sorg og Suk gaaer da allerede tidligt gjennem hendes Hjerte . Herren give , at en oprigtig Foremng mellem begge Familier allevegne maa sinde Sted ! Dog not herom . Vi maae endnu omtale nogle Andre , som ogfaa have sine Fordringer til Brylluppet . Overalt i Christenheden hvor man har feiret en rigtig Glcedesfest , har man ogsaa betcenkt de <b>Fattig</b> . Glceden udvider Hjertet . Glceden ihukommer ogsaa dem , som ikke glc.de sig paa den Dag- Gud Fader har indbudt <b>Fattige</b> , Halte , Kroblinger og Vlmde til sin Sons Bryllups Vi ere selv sådanne Gaster Saa ville vi da ikke glemme de <b>Fattige</b> i vor Glcede Paa <b>Land</b> stal der tidligere have vceret Skik , at man hensatte et eg Fad , hvorpaa man lagde Noget af hver Ret til dem Dette kaldte man Guds Deel . den Deel , Han ved de <b>Fattige</b> fahentede til sig . Dette er vel nu gaaet af Brug ; men man kan dog endnu give Gud Hans Deel i Penge og andre Gaver . Ogsaa de <b>Fattige</b> stulle beds for det unge Par og latte Gud for den Naade , Han han beviist det . Paa denne

<h4>1120</h4>Vistnok maa ogsaa mangen <b>Familie</b> bekjends : „ Vinteren er for os den ulykkeligste Tid " , Noget , som forovrigt er dens egen Skyld . Hvor nemlig ikke Guds Fred boer i Huset , der hersker der mere Kiv og Trcette om Vinteren end om Zommeren . Om Sommeren har man lettere for at gaae afveien for hinanden . Hertil kommer desuden , at trangt Kaar foles mere trykkende om Vinteren . Som en Fslge af at man ikke boerer sin Nod i Herren skyder den Ene Skylden paa den Anden . Man kives om Brod , om Klceder , om Lys , om Varme og andre Ting . Men i Grunden kives man , fordi man ikke finder nogen Smag i Livsens Brod , og fordi det er saa morkt og koldt i Ens Indre . En Vinter uden Herren er et Helvede i Hnset ; en Vinter med Ham er en dyrebar Naadestid og en Haandvenge vaa den salige Hvile i Gud . Og som for det huuslige Liv , saaledes har den ogsaa sin Betydning for Folke- og Menighedslivet . „ Fred paa Jorden ! " sang Englene ved Bethlehem , I een Forstand tiltvinger Gud sig ncesten med Magt en overfladisk Fred : Norden . Krigene maae nemlig da tie , og imidlertid havde stridende Parter Tid til at tcenke over sin Synd og Skyld , over sine udsvede Rcedselsgjerninger og over Freden . Gud driver Menneskene til Fred , men endnu mere til Barmhjer tighed . Det kan ikke negtes , at Noden hviler tungere 2 e <b>Fattige</b> om Vinteren end om Sommeren . Da har den christsligs Kjcerligbed den mest ndstrakte Mark for sin Virksomhed. Da bor den ikke glemme at gjore vel og at dele ; da bor den besoge Faderlose og Enker i deres Trceng fel , aabne sit Huns for de Elendige , kloede de Nogns og vrcrdike Evangelium for de <b>Fattige</b> . Naar AK ligger stivnet og dodt , naar Marken hverken har Grces eller Blomster , da bor den fra Eden overplantede Blomst : den af Herren jodle Kjcerlighed , iscerdeleshed blomstre . Kærlighed oo Barmhjertighed ere Vedbenden og Sneklokken i de bave / Da er det Vinter og dog ikke Vinter ; da er det koldl og dog ikke koldt . Paa den Maade vil Gud , der ogsaa ha > skabt Vinteren , bruge den , som al Trcengsel til stn AZre og nn Menigheds Gavn og Bedste . Nu vel , hcere Lcesere , prn ser da Herren for alle Hans Gjerninger og Indretninger , vriser Ham , fordi Han ikke lader Sommer og Vinter , Fron og Hede , <b>Sced</b> og Host , Dag og Nat ophore ! priser Ham under Korset og i gode Dage ! men benytter ogsaa ielen , de morke Dage og lange Vinteraftener re alvorligt til Eders Frelse og til Guds Forherligelse i Menigheden . Lader ikke Lcengselen efter det <b>Land</b> uddse , hvor der ine lcenger ' er nogen Storm i Sjelen , og hvor de unge lud paa Livstrceet ikke mere bortfryss og henvisne ! Herre vor

<h4>1210</h4>Saaledes ere de Christne Kongebarn , der vandre paa Jorden, men leve i Himmelen , der ere Fremmede og Udlcendinger herneden , men med sit Hjerte og Haab ere i Hjemmet, der maae dsie Foragt og Fornedrelse af Menneskene , men dog med sin Tro og VM opretholds og regjere Verden. Dette er Guds Vsrns kongelige Stand . Salig er den , som lever i en sikker Forvisning om og daglig Ud ovelse af denne Herlighed og Hsihed . Om hvilke Mennesker og hvilke Familier sige vi nu som . Christne , at de ere gaaede tilbage ? Ikke om de forarmede Noomi kommer tilligemed sin Svigerdatter Nuth tilbage fra Moabiternes <b>Land</b> . Hendes Gods har Dyrtiden bersvet hende ; sin Mand og sine Ssnner har hun begravet i det fremmede <b>Land</b> ; hun er bleven saa haardt hjemsøgt , at hun ikke Icenger vil kaldes Noomi , Inde , men Mara , Bitterhed . Og dog er hun endnu ovenpaa ; thi hun har endnu ikke sluppet sin Gudsfrygt og sit Haab til Herren . Den gamle Tobias er en landflygtig Mand tilligemed sin <b>Familie</b> . Han har mistet sit <b>Fcedreland</b> og sit Arvegods ; raat Voldsherredømme lcegger sin lernhaand paa hans og hans Folks Nakke ; hans Vines Lys er udstukket , og hans <b>Kone</b> erncerer ham ved at spinde . Ikkedestomindre siger han til sin Son : „ Frygt ikke , mit Barn , fordi vi ere blevne <b>fattige</b> ; du har Meget , om du frygter Gud og viger fra al Synd og gjsr det , fom er behageligt for Ham . " Udvortes ere disse Mennesker komne paa Knceerne ; men indvortes staaer den Gamle hoit over det vanartige Hedningefolk , blandt hvilket han lever som Fange . lofef og Maria ere efter sin Herkomst cegte Kongebarn , fsdte af den fornemste Slcegt i Verden . Men Josef er dog ikke Andet end en Tommermand i den lille foragtede By Nazareth , og i Bethlehem kan dette Par kun sinde Herberge i en Stald . Om Maria og hendes nyfpdte Barn siger en gammel ranger :

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100803012'target=_>, 1847, Rescripter, Resolutioner og Collegial-Breve for Kongeriget Norge i Tidsrummet fra 1660-1813</a><h4>5499</h4>bestandig under Anforsel af ferstc Larer , Kirkesangeren ' ) . § 30. Da denne Skole egentligcn sial vare for Byens <b>fattige</b> og foraldrelose Born , saa ber Fattigskolens Casse anskaffe fornodne Boger , Skrivematcrialier o . ' s . v. § 31 . For saadanne <b>fattige</b> Bern , der " ere i den Alder , at de kunne vare deres Faraldre , eller dem , de af Fattig-Commissioncn ere betroede» til Hjalp i disses Haandtering eller Naringsvci , stulle Larclimcrnc saalcdes indsirankes, at Drengene undervises om Formiddagen og Pigernc om Eftcrmibdagen ; dog lv ' r håves Indseende med , at derved ei gives Anledning til Lcddiggang . § 32. Undcrviisnings- Gjenstandenc i denne Skole blivc ^ : Indcnads-Lasning , samt Skrivning og Lasning sra de forste Elementer af , Religion med <b>Moral</b> og Bibelhistorie , de friste Grunde af Geographie og Historie , isar <b>Fedrelandets</b> , saa og det Nodvcndigstc af Naturlare og Naturhistorie , for at betrygge Vernene mod den saa gangse og skadelige Overtro . Med disse Gjenstande stal , om muligt , forenes en elemcntar Undcrviisning for Born af alle Classer , saavidt Hovedoicmedet ide Fattiges Unbervijsning ilke derved skulde forstiles . Finder Commissioncn , ester at Stiftelsen er sat i Gang , en saadan elemcntar Underviisning forccnbar med Hovedeicmcdet, da stal for denne ydcs en vassende Betaling . § 33. Hver Dags Underviisning bcgyndes , ligesom i Borgersiolen , med B ? n og Sang , hvorved Lccrerne have at paasee , at Vernene iagttagc den tilbcrlige Stilhcd og Opmartsomhcd , ligesom de da isar bor erindres om den Wrbedighed , de siylre den Allcstcdsnarvarende . § 34 . I <b>Fattig</b> > som i Borgcrsiolen inddelcs Vernene i Classer , i Forhold til deres Kundstabcr , og undervises , efter narmere Overeenskomst imellem Larerne , saaledes , at enhver Larer doccicr i sit Fag . Det bor og her iagttagcs , at hvert Kjen sidder saa meget muligt adstilt , paa det at ei sial gives Anledning til Usemmeligheder og Uordencr . § 35. Viser noget Barn saa stet Opfersel , at Lareines Formaninger , Irettesattclser eller lemfaldige Straffe ilke kunne virke Forbedring , da bor det anklages for Commissionen , som tåger passende Forholdsregler , enten ved Ude " luklelsc fra Skolens Beneficia , eller ved passende legemlige . Straffe . , § 36 . Ethvert Barn bor mode ordentlig og lil bestemt Tid i Skolen , og for Forsommelse heri alvorlige » ircttcsattes; virker ikke dette Forbedring , da bor dets Foraldre ellrr narmeste Foresatte staae til Ansvar derfor , og saadan gjentagcn Skjodcsleshcd anmeldes for Skolc-Commissionen eller den af dens Lemmer , som har Inspeclion i Ugen . § 37. Vernenes Fremgang bliver undcrftgt ved Eiamen , som holdes hver Juni Maancd ester Bekjcndtgjorclse fra Pradikestolen . 8 38. Hver af Skolens Larcre sial holde sin Journal over Vernenes Fremgang eller Forsommelser, Opferscl , Sygdomstilfaldc m . v. ' , som bliver at efterscc af dem , det tilfalder at have Opsyn med Skolen . § 39. Om Skole- eller Lasetiden gjalder dct , Samme her , som for Borgersiolen er fastsat . § 40. Da disse Larere formodentlig kunne have nogen Tid tilovcrs til clementar Undcrviisning for de Born , som ville sege Borgerskolen , uden at de Fattiges Nndcrviisning derved vil lide , bor de ikke negtes Frihed til at forbedre deres Kaar ved <b>privat</b> Underviisning udenfor Skoletimerne . § 41. levrigt ber de for Borgersiolm fastsatte Bestemmelser , angaaende Larerncs Forhold , indtraffende Sygdomme eller Ferier , ogsaa vare gjaldcnde for denne Skole . § 42. Forste Larer nyder , foruden Kirkcsangertjcnestens Indkomster , 100 Rdlr . aarligen af Fajtig-Casscns Fond . Anden Larer nydcr aarligcn 200 Rdlr . , som indkomme saaledes : af Fattigskolens Fond 100 Ndlr . , af Skiens Hospital 30 Rd . , ved Ligning 70 Rd . DesAden nyder han frit Brande og Logis i Fattigskolen; men derfor paaligger ham ogsaa , foruden hans Pligtcr som Larcr , hver Helligdag at forelase en Vradiken over Dagens Text , af en eller anden god Forfatter , for H ospitalets ^ ) Lemmer . § 43. Ferste Larer ved Borgersiolen , der , i Folge § 3 , sial vare ordineret Catechet, vil Kongen , i sin Tid , Selv bestikke ; til Besattelse af de evrige Larerpladse overlades det Commissionen at gjere Forslag for vedkommende Autoriteter , som da eraminere de forestagne Subjecter , og efter de befundne Omstandighedcr bestikke dem .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092403020'target=_>Ahlfeld, Friedrich, 1865, Menneskelivet i Guds Ords Lys</a><h4>586</h4>hver ussel Syuter , ligesom Han endnu stedfe tjener os i den Hellig-Acmd med al sin Kjerlighed og alle sine hoihellige Nnndcgnver . — San giv dn ogsnn os Trostab og Mmyghet » til som gode Huusholdere over Guds mangeslags Naade at tjene hverandre med den Naategave , Enhver af os har facet . Men forbarm Dig iferdeleshet » over alle dem , der indtrede forn Tjenestetyende , Dreng eller <b>Pige</b> , i sin Nestes Huus . Lad dem tjene som for dit Ansigt . Lad dem forst og fremst beljende : „ Se , jeg er Herrens Tjener , Herrens <b>Tjenerinde</b> . " Giv dem Wmyghed , Sanddruhed og Trostab , saa at de ikke soge sit Eget , men omgaces til dit sulte Velbehag og af yderste Evne soge at fremme sit Huus ^ s Bedste . Lad dem med sine Hjerter slutte sig til <b>Familien</b> ; lad ogsaa dem hore til dens lille Menighed . Giv dem Alle Naade til af sit Herstnb nt annamme nogen Velsignelse for det evige Liv og til ligeledes paa sin Side at kunne vere sit Herstab til nogen Velsignelse . Det ville Du gjore for Jesu Christi Skyld ! Amen . Uden gjensidig broderlig Tjeneste er Livet ikke tenkelig ! eller muligt . Christi Kirke er det store Legeme , og paa det ere vi Alle Lemmer . Det ene Lem maa tjene det andet med sin Naategave . Den Kloge maa tjene sine Brodre med sine Naad , den Sterke med sin Styrke , den Nige med sin Rigdom. Den <b>Fattige</b> tjener den Rige igjen med sine Krefter . Men Alle stulle tjene hverandre med den af Herren fodte Kjerlighed . — Lige fra den grace Oldtid har der bestandig veret Mennesker , som have givet sig ind i Andres Huse for at tjene og hjelpe dem . Elieser tjener Abraham ; Jakob tjener Laban ; losva tjener Moses . Ved Siden af saadcmne frie Tjenere gaves der ogsaa Tjenere og Tjenesteviger i Ordets haardeste Betydning , nemlig Slaver eller Trelle , Slavinder eller Trelkvinder . Til disse var det Paulus talte i vor Text . Saadcmne sindes gndstelov ikke lenger i vort <b>Land</b> . Deres blotte Navn indjager Skrek . Dog er det ikke blot for denne haarde , ufrivillige Tjeneste , man frygter ; Enhver saae i vor Tid helst , at han var fri for at tjene . Selvralldigheten og Hovmodet ere blevne mcegtige blandt Menneskene . lordbrugere og Driftsherrer , der have tungt Arbeide , klage over at de ikke lenger kunne saae Tjenestefolk. Alle gribe efter lettere og , som de troe , mere lonnende Beskjeftigelse . Alle ville allerede i sin Ungdom vere frie og selvstendige . Unge Karte og <b>Piger</b> ville heller arbeide i Fabrikker end gaae i Tjeneste . O , de vide endnn ikke , hvilket langt storre Verd det huuslige Liv har end Livet blandt den store fra- og tilstrommende Wengte , der bringer Synder fra alle Verdens Kanter sammen nuter eet

<h4>1172</h4>Snnledes ere de Christne Kongeborn , der vandre paa Jorden, men leve i Himmelen , der ere Fremmede og Udlendinger herneden , men med sit Hjerte og Haab ere i <b>Hjemmet</b> , der maae doe Foragt og Fornedrelse af Menneskene, men dog med sin Tro og Bon opretholde og regjere Verden . Dette er Guds Boms kongelige Stand . Salig er den , som lever i en sikker Forvisning om og daglig Udovelse af denne Herlighed og Hoihed . Om hvilke Mennesker og hvilke Familier sige vi nu som Christne , at det er gaaet tilbage med dem ? Ikke om de forarmede . Noomi kommer tilligemed sin Svigerdatter Ruth tilbage fra Moo.biteru . es <b>Land</b> . Hendes Gods har Dyrtiden stilt hende ved ; sin Mand og sine Sonner har hnn begravet i det fremmede <b>Land</b> ; hun er bleven faa haardt hjemsogt , at hun itte lenger vil kaldes Noomi , Inde , men Mara , Bitterhet » . Og dog er hun endnn ovenpaa ; thi hnn hnr endnu itte fluppet sin Gudsfrygt og sit Hnnb til Herren . — Den gamle Tobias er en lcmdflygtig Mand tilligemed sin <b>Familie</b> . Han har mistet sit Fedreland og sit Arvegods ; raat Voldsherredomme legger sin lernhacmd paa hans og hans Folks Natte ; hans Omes Lys er ud sluttet og hans <b>Kone</b> ernerer ham ved at svinte . Ittedestomindre siger han til sin Son : „ Frygt itte , mit Barn , fordi vi ere blevne <b>fattige</b> ; du har Meget , om du frygter Gud og viger fra al Synd og gjor det , forn er behageligt for Ham . " Udvortes ere disse Mennesker komne paa Kneerne ; men indvortes staner den Gamle hoit over det vcmartige Hedningefolk , blandt hvilket han lever som Fange . — Josef og Maria ere efter sin Herkomst egte Kongeborn , fodte af den fornemste Slegt i Verden . Men Josef er dog itte Andet end en Tommermcmd i den lille foragtete By Nazareth , og i Bethlehem kan dette Par tun sinde Herberge i en Stald . Om Maria og hendes nyfodte Varn siger en gammel Sanger :

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014061108162'target=_>Smiles, Samuel, 1876, Karakteren</a><h4>178</h4>Men Athen havde to skjebnesvangre svage Punkter : Dets Borgere kjendte ikke til noget sandt Familieliv eller <b>Hjem</b> , og dets <b>Samfund</b> talte flere Slaver end frie Mcend . Dets offenlige Mcend vare lpfe , om ikke fordærvede , i <b>moralsk</b> Henseende . Dets <b>Kvinder</b> , selv de fuldkomneste blandt disse , vare ukydste . Derfor var dets Fald uundgaaeligt , og det gik endog pludseligere for sig , end i sin Tid dets Opsving .

<h4>344</h4>Kvinderne ere det modsatte ; og nåar , som vi mene Tilfceldet er , et Folks <b>moralske</b> og scedelige Tilstand hovedsagelig beror paa Opdragelsen i <b>Hjemmet</b> , da maa Kvuldens Udvikling anfts for en Sag as national Betydning . Ikke alene vil Mandens <b>moralske</b> Karakter men ogsaa hans Sjcels slurke finde sin bedste Bevogter og Stolte i <b>Kvindens</b> <b>moralske</b> Renhed og aandelige Udvikling ; men jo mere begges Magt er udviklet , jo mere harmonisk og velordnet vil <b>Samfundet</b> vcere des sikrere og tryggere dets Fremgang og Foradling .

<h4>918</h4>Som vi alt have paavist , er <b>Hjemmet</b> den forste og den bedste Dannelfesanstalt for <b>moralsk</b> Tugt ; efter dette kommer Skolen , og dernæst Verden , det praktiske Livs store Skole . Hvert af disfe Stadier paa Udviklingens Vej er en Forberedelse for det ncestpaasolgende , og hvad Manden eller <b>Kvinden</b> bliver til , afhcenger for storfte Delen af det , der er gaaet forud . Have Mennesker hverken nydt Hjemmets eller Skolens Velsignelser , har man ladet dem volfe op uden Dannelfe , Undervisning eller Tilsyn , da Gud naade dem selv og det <b>Samfund</b> , de hore til !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010022412001'target=_>, 1845, Kongelige Rescripter, Resolutioner og Collegial-Breve for Norge</a><h4>2731</h4>46 Confirm , paa det af Andreas Rierrumgaard , Klokker til Domkirken i Vergen , under 4 Jan . 1800 oprettede Gavebrev paa 550 Rd . , foreget af Kongen med 50 Rd . , i Stedet for ansogte Eftergivelse af Qvart-Procent- Skatten ) til 6 a 8 eller flere <b>fattige</b> Berns llnderviisning i danst * ) Lcesning , Bsrne-Lcerdom og Skrivning , hvilke han selv nogle Aar hidtil har holdet i <b>privat</b> Skole ved Nostet , hvorfra Veien falder for lang til at ssge de Publiqve ; hvilken Summa fer hans dgdelige Afgang sial aflcegges og overdrages til 2 vederheftige Mcend som Crecutores , dem han maatte udncevne , der blive ansvarlige for og bestyre det Legerede saaledes : At af Sammes Renter udbetales quartaliter eller hvert halve Aar Skole- Lsnnen , og Resten af Renterne anvendes til Bred , Skoe , Strsmper eller andre stige Nsdtorftigheder for Vsrnene ( i Stedet for Stromper kan og kjsbes Garn , som <b>Pigerne</b> selv strikke , hvortil de og bsr holdes ) . Disse Udgifter og Vernenes Antal , som ei ber vcere mindre end 6 , bestemme de udncevnte Erecutores og Bestyrere , nogenledes i Forhold med Indtcrgten , samt antage andre Born i deres Sted , som efter dcrrs Skjsn enten gaae ud af Skolen , eller scettes hen i de publiqve Lcese-Skoler til videre Underviisning . De scette og Vernene hos en Skoleholder eller Lceremoder , hvor de bedft oplceres og holdes til <b>Scedelighed</b> . Skulde en af de udncevnte Bestyrere bortdste, antager den Loengstlevende cn anden paalidelig Mand , helst af Domkirkens faste Prcrster , i den Afdsdes Sted . Efter de udncrvnte Erecutorers Ded , ifald han da ei lever , overdrages dette Legats Ansvar og Bestyrelse til Inspectorerne for Domkirkens saakaldte Graa- Skole dog under Directionens Opsyn , hvilken vilde bestikke en vederheftig Mand til , at vcere ansvarlig for det Legerede , og indcassere Renterne . Han forbeholder sig selv Ret og * ) Det er : " Norsk . " " ) W - . Conf . 11 April 1806.

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081903028'target=_>Hippel, Theodor Gottlieb von, 1861, Om Ægteskabet</a><h4>358</h4>hedde i vor Oversettelse saaledes : Hvo kan finde en duelig O.vinde ? hun er langt mere vcerd end Perler . Hendes Mands Hjerte tor forlade sig paa hende , og ham stal ikke fattes Fode . Hun gjor ham Godt og intet Ondt , i alle sit Livs Dage . Hun soger ester Uld og Hor , og arbeider gjerne med sine Hcender . Hun er ligesom en Kjsbmands Skib ; hun lader komme sit Brod langt fra . Og hun staaer op , naar det endnu er Nat , og giver sit Huus Spise , og sine unge <b>Piger</b> deres bestikkede Deel . Hun tcenker paa en Ager , og faaer den ; hun planter en Viingaard as sine Hccnders Frugt . Huu ombinder sine Lcender med Styrke , og bestyrker sine Arme . Hun fornemmer , at hendes Kjobmandstab er godt ; hendes Lygte skal ikke udslukkes om Natten . Hun udstrcekker sine Hcender til Tenen , og hendes Hcender tåge fat paa en Nok . Hun udbreder sin Haand til den Elendige og udstrcekker sine Hcender til den <b>Fattige</b> . Hun frygter ikke for sit Huus for Snee ; thi alt hendes Huus-Folk er beklcedt med dobbelte Klceder . Hun gjor sig Tapeter ; hvidt Linned og Purpur er hendes Klceder . Hendes Mand er navnkundig i Portene , naar han sidder hos de 3 Eldste i Landet . Hun gjor kosteligt Liinkloede cg scelger det , og overantvorder Kjobuianden Belter . Styrke og Herlighed er hendes Klcedebon , og hun leer til den efterkommende Tid . Hun oplader sin Mund med Viisdom , og Mifkundheds Lov er paa hendes Tunge . Hun seer , hvorledes det gaacr til i hendes Huus , og ceder ikke Vrod med Ladhed . Hendes Sonner staae op og prise hende salig ; hendes Huusbond roser hende ogsaa , sigende : Mange Dottre forhverve Formue ; men Du , Du kom op over dem Allesammen. Åndelig er Vedragelighed , og Deilighed er Forfwngelighed: men en Ovinde , i hvilken der er Herrens Frygt , hun stal roses . Giver hende as hendes Hcenders Frugt , og hendes Gjerninger stulle prise hende i Portene . En saa fortreffelig Beskrivelse af en <b>Hustru</b> skulde man vel ligesaalidt vente af Kong Salomon , som at Lex Papia Popcea forer Navn efter to Folk , som hverken havde <b>Kone</b> eller Barn ; men netop derfor ere Begge mig saa meget mere lcererige . Hvem kan ncegte , at Mgtestaber jo overordentligt lette Regjeringen af <b>Land</b> og Folk ; da Huusfader og <b>Huusmoder</b> beklcede et Slags Fredsdommer-

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009082700002'target=_>Taine, Hippolyte, 1877, engelske Literaturs Historie</a><h4>80</h4>og Forholdene vare ganske vist anlagte herpaa , man har sjeldent seet mere Ulykke og mere Talent samlede hos et Menneske . Han blev født i Januar 1709 , under en skotsk Vinters strænge Kulde , i en Leerhytte , som hans Fader , en <b>fattig</b> Forpagter i Ayshire , selv havde bygget : en sørgelig Stilling , et sørgeligt <b>Land</b> , en sørgelig Hytte . Gavlen faldt ind nogle Dage efter hans Fødsel , og hans Moder blev nødt til midt under Stormen at gaae med ham til en Nabo , for at søge Ly . Det er haardt at fødes i et saadant <b>Land</b> , der er saa koldt der , at jeg engang i Juli i Glasgow , i smukt Solskinsveir havde god Brug for min Kappe . Det er en daarlig Jord , der er enMængde nøgne Høie , hvor Hosten ofte slaaer feil . Burns ' s Fader var allerede gammel og havde ikke meget Andet end sine Arme at støtte sig til , han sad for altfor høi en Forpagtningsafgift, han havde syv Born at sørge for , han slog sig igjennem med Sparsommelighed eller rettere ved at holde Faste , og boede som Eneboer , for ikke at fristes til at gjøre Udgifter . » Flere Aar igjennem var fersk ! Kjød en ukjendt Ting i Huset « . Robert gik med bart Hoved og bare Fødder , da han var tretten Aar gammel tærskede han Kora , da han var femten Aar » gjorde han Tjeneste som første Avlskarl « . <b>Familien</b> gjorde Alting selv , de holdt hverken Karl eller <b>Pige</b> , de havde næsten Ingenting at spise og de havde altfor meget at bestille . » Lige til mit sextende Aar « , siger Burns , » levede jeg saaledes i en Eneboers glædeløse Tungsind , under en Galeislaves uophørlige Slid « . Hans Ryg blev krum , hans Sind blev mørkt , næsten hver Aften var hans Hoved smertende og tungt , i senere Dage fik han Hjertebanken , og om Natten , naar han var i Seng , var det ham , som skulde han kvæles , og han var nærved at besvime . » Den sjælelige Ængstelse vi følte « , siger hans Broder , » var meget stor « . Faderen blev gammel , hans graa Hoved , hans furede Pande , hans indfaldne Kinder , hans høie Skikkelse , der

<h4>1306</h4>Man seer , at dette er en ny Art Hykleri ; Lasterne skifte hvert Aarhundrede sammen med Dyderne . Den praktiske Aand er ligesaavel engelsk som den <b>moralske</b> Aand ; dette Folk har handlet , arbeidet og regjeret sig selv saalænge , at det tilsidst har faaet Lyst og Evne til Forretninger ; det er derfor de betragte os som Bern og Galninge . Naar denne Retning overdrives , ødelægger den Fantasien og Følelsen , man bliver en Regneraaskine, i hvem der kun lever Kjensgjerninger og Tal , man nægter Aandens Liv og Hjertets Glæder , man seer ikke andet i Verden end Indtægt og Tab , man bliver haard , barsk , begjærlig og gjerrig , man behandler Menneskene som Redskaber , og saa kommer der en Dag , hvor man er bleven lutter Handelsmand , Bankier og Statistiker , og hvor man ikke længere er Menneske . Dickens har givet mangfoldige Skildringer af den kolde praktiske Mand : Ralph Nickleby , Scroogs , Antony Chuzzlewitt , Jonas , Alderman Cute , Mr . Murdstone og hans Søster , Bounderby , Tom Gradgrind , de findes i alle hans Romaner . Nogle have faaet Præget ved Opdragelsen , andre have det fra Naturen af , men de ere alle modbydelige , thi de gjøre det alle til deres Opgave , at spotte og tilintetgjøre Godheden, Medfølelsen , Medlidenheden , de uegennyttige Følelser , de religiøse Stemninger , Fantasiens Begeistring , kort sagt , alt det Skjønne hos Mennesket . De undertrykke Børaene , de slaae <b>Kvinderne</b> , de sulte de <b>Fattige</b> , de forhaane de Ulykkelige . De bedste af dem ere glatte Jernautomater , der regelret udfore de Pligter , Loven paalægger dem , og ikke vide , hvormegen Smerte- de paadrage andre . Den Slags Mennesker findes ikke i vort <b>Fædreland</b> ; deres

<h4>1339</h4>Saa felger det da af sig selv , at Dickens forherliger dem . Det er ikke noget nyt i Frankrig ; Eugen Sue ' s Bomaner have givet os mere end eet Exempel derpaa, og det er en Paastand , der naaer tilbage til Rousseau, nåen under den engelske Forfatters Hænder har den faaet en særlig Styrke . Hans Helte ere beundringsværdigt hensynsfulde og opoffrende , der er ikke andet simpelt ved dem end deres Udtale , ellers ere de lutter Adel og delmodighed. Vi see én Gjøgler forlade sin Datter , hans eneste Glæde , af Frygt for at skade hende i Noget . En ung <b>Kvinde</b> , der elsker en Mand , som elsker hende igjen , opoffrer sig for at frelse denne Mands uværdige <b>Hustru</b> ; Handen døer , men hun vedbliver af ren Selvfornægtelse at pleie denne nedværdigede Skabning . En <b>fattig</b> Fragtmand , der troer , at hans <b>Kone</b> er ham utro , erklærer hende med hei Køst for uskyldig , og istedetfor Hævn tænker han kun paa at overvælde hende med Ømhed og Godhed . Der er ifølge Dickens ingen , der saa levende som dem føler , hvor stor en Lykke det er at elske og blive elsket , og hvor ren en Glæde Familielivet kan give . Der er ingen , der har saa megen Medlidenhed med disse stakkels , vanskabte og svage Væsener , som de saa ofte bringe til Verden , Jog som kun synes at fødes for at døe . Der er ingen , der har en sundere og mere übøielig <b>moralsk</b> Sands , ja , jeg maa endogsaa tilstaa , at disse Helte hos Dickens ere ulykkelige nok til at ligne de vrede Fædre i vore Melodramer . Da den gamle Peggotty faaer at vide , at hans Søsterdatter er blevet forført , giver han sig paa Vei med Stokken i Haanden , og drager gjennem Frankrig , Tydskland og Italien , for at finde hende og føre hende tilbage til hendes Pligt . Men fremfor alt have de en engelsk Følelse , og som mangler hos os , de ere nemlig Kristne ; det er ikke blot som hos os <b>Kvinderne</b> , der tåge deres Tilflugt til Forestillingen om et andet Liv , ogsaa Mændene tæuke derpaa . I dette <b>Land</b> , hvor der er saamange Sekter , og

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092203003'target=_>, 1859, Almindelig Norsk Lovsamling</a><h4>5429</h4>ansees nodvendige eller gavnlige for Fattig-Vesenets Oiemed . 4 t Skolegangens Oiemed bor vere : at Vernene ved deres Sjels- og Legems-Kreftcrs rigtige Udvikling og Anvendelse kunne dannes til oplyste , gode , arbeidsomme Mennester og Borgere , at de tll den Ende tidligen hindres fra Lediggang og Laster , og venncs til <b>Sedelighet</b> » , Orden og Nenlighed ; at de anferes til saadan Kundstab , som er nedvendig og gavnlig fer Mennesket og Borgeren i Almindelighed ; og at de undervises og eves i saadant Arbeide , hvorved de i Fremtiden , enten i Mangel af andet Nerings-Arbeide kunde fortjene deres Underholdning , eller nyttigen beskjeftige sig i ledige Timer . 42. Undervisningen i Skolerne stal altsaa vere delt imellem Vldenstab , Haand-Arbeide og nyttige Legemsøvelser. 43. Ved Videnstab forstaaes ikke her det , som henhorer til blot Lerdom , eller blot spekulativ Kundstab i egentlig Forstand , men det , som for den menige Almue er fornedent, fatteligt og brugbart , saasom : Lesning , Skrivning , Regning , Religion og Moral, almindelig Kundsiab om Naturen , dens Indretninger og Virkninger ; om borgerlige Indretninger og Pligter , om de vigtigste Sundhcds-Regler , om <b>Fedrelandets</b> Forfatning og Veliggenhed og deslige , alt efter korte og simple Lerebeger , som dertil kunne velges, og med Hensyn til den Forskjel , som ber gjeres i Undervisning for Drenge og <b>Piger</b> . Ogsaa serges for , at Bornene anferes til rigtig og harmonisk Sang , iser Choraler og gode Folkesange . 44. Haond-Arbeidet ber vere af det Slags , som i det daglige Liv kan for Almuens Folk vere almindeligt og gavnligt , og som for Vernene efter Forskjellighet » af Kjon , Alder og Krefter findes passende , samt hvortil i Skolebygningerne kan vere Leilighed og Rum , saasom : at pille Bomuld , at karde Uld og Bomuld , at plukke og spinde Uld , Bomuld , Her , Hamp og Blaar , at tvinde , haspe , vinde , spole , at binde og strikke , at kniple , at lappe paa Kleder , at stoppe Stremper , at knytte Fiskegarn, at veve Bendler og uldne Baand , at gjore Hegter og Fistekroge , stette Kurve og Hatte , at forferdige allehacmde Legetei og andet Nurnberger-Kram , ug andet mere ; og for <b>Pigerne</b> tillige , at tildanne og sy Linned og Kleder m . v. Ved Arbeidets afverlende Afdeling sees blandt andet ogsaa derpaa , at Legems-Krefterne styrkes tilligemed eves . 4. ? . Til at give Undervisning i Haand-Arbeide ansettes ved hver af Fattig- Vesenets Skoler en Leremoder , som anvises den fornedne Plads i Skolebygningen ; og forsaavidt Leremoderen ei kunde gjore det , soger Direktionen at forene sig med een eller stere Arbeidere , som maatte besidde de fornedne Ferdigheder , om visse Timer daglig enten hjemme hus sig selv eller i Skolebygningerne at give Undervisning i Haand-Arbeide . 4 V . Legems-Ovelser ber vere af det Slags , som kunde bidrage til den Styrke , Smidighet» og Behendighet » , som i den arbeidende Klasse ere nodvendige og nyttige . Drengebornene ber om Sommeren ofte feres ud at bade sig i Langen , og tillige gives Undervisning i at svemme . For at dette med Sikkerhed kan ste , treffes de fornodne Foranstaltninger. Til at undervise i Legemsouelser antages helst en duelig Under-Ofsicier , som er oplert i Gymnastiken , da han tillige kunde eve de eldre Drengebern i at erercere. 47. Ingen Understettelse fra Fattig-Vesenet gives til Underholdning for <b>fattige</b> Vern , uden under det udtrykkelige Villaar , at Bornene stulle stadigen sege den Undervisnings og Arbeids-Skole , hvortil de henvises ; og findes Almisse-Lemmer eller Andre , som ved Fattig-Vesenet ere indstrevne , forsemmelige i at tilholde deres Bern at soge Skolerne , eller de selv afholde dem derfra , advares de forst , og nåar Advarsler ei frngte , ansees de efter § lUO . Er det Vernenes egen Skyld , at de udeblive , ansees de medde Straffe , som Skole Reglementerne nermere for saadan Forseelse fastsette . 48. Fra sormeldte offentlige Skolegang kan ug bor ikke , Sygdoms-Tilfelde undtagen , ste Fritogelse , med mindre Direktionen finder gyldige Grunde til saadant at tilstede for en Tid ; men i alle Tilfelde , hvor Undtagelse saaledes tilstedes , ber omhyggeligen serges for , at de Bern , som i den bestemte Tid ikke mede i Skolerne , dog ikke savne den Undervisning og Anfersel til Arbeide , som er Skolegangens Oiemed . 4 » . Naar <b>Fattige</b> , der have Bern , som ber sege Skolen , bo saa langt fra den nermeste Skole , at Vernene tilvisse Tider af Aaret ikke stadigen funne indfinde sig der , ferger Direktionen , saa vidt muligt , for at disse Familier eller blot Vernene flyttes ind til Bopele , som ligge beleiligere , iltt . For saa vidt det er gjerligt , serges fur , at Drengenes og <b>Pigernes</b> Undervisning adstilles , saaledes , at nåar Drengene undervises af Skolelereren , <b>Pigerne</b> anfores af Leremoderen til Haand-Arbeide , og omvendt ; og nåar Saadant er forbundet ^ ned for megen Vanskelighet » , Drenge og <b>Piger</b> holdes saavidt muligt i serskilte Verelser . ilt . Hvor tidlig om Morgenen Vernene stulle mode i Skolerne , hvorledes Tiden , medens de ere der , inddeles , imellem afverlende gradvis fremgaaende og efter deres Evner afpasset Undervisning i Kundstab og Anfersel eller Ouelse i Arbeide , med videre ; hvilke Lere-Methoder og Lerebeger stulle indferes , for rigtigen at udvikle deres Evner , og Veilede mere til Selvtenken end til Udenad-Lesning ; hvorledes Bornene fordeles i Klasser: dette bliver i Reglementer for Skolerne i Almindelighed eller for hver iser efter

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010020112014'target=_>Sundt, Eilert, 1870, Om fattigforholdene i Christiania</a><h4>8241</h4>Stadsfysikus Bidenkap : Det var , forsaavidt jeg har forstaaet Koniiteens Mening , om Sygepleiens Organisation , jeg skulde meddele nogle Oplysninger. Endel saadanne findes allerede i Sundts Bog om Fattigforholdene i Christiania . Der er taget med baade Beretning om Lægetilsynet og om Sygehusforpleiningen ( dog mindre om denne Sidste ) samt om de almindeligste Sygdomme blandt de <b>Fattige</b> og om forskjellige Ting , som jeg altsaa ikke behøver at gjentage . Ligeledes vil der i den Forklaring , som er udarbeidet af en Komite inden Fattigkommissionen , hvoraf jeg var Medlem , og som blev afgivet den 30 te November 1869 , findes en kort Fremstilling af Organisationen af Sygepleien . Jeg skal derfor henvise til disse trykte Dokumenter , som jeg vil supplere med , hvad jeg har at sige , som da væsentlig er nogle yderligere statistiske Oplysninger . Angaaende Organisationen af den offentlige Sygepleie i sin Almindelighed her i Christiania skal jeg oplyse , at denne er paabyrdet , kan jeg gjerne sige , Fattigkommissionen i en meget videre Udstrækning , end man kunde forudsætte efter Loven og efter Forholdene paa andre Steder hertillands. Fattigvæsenet har nemlig overtaget ikke alene Behandlingen og Forpleiningen af <b>fattige</b> Syge i <b>Hjemmet</b> og paa Sygehusene , men ogsaa Indredningen og Driften af Epidemi- Lazarether , og andre Sygehuse , som er bestemt for specielle Sygdomme , der maa skjøttes af det Offentlige , ikke paa Grund af at de Syge ere <b>Fattige</b> , eller ikke væsentlig ialfald af den Grund , men fordi Sygdommen er faretruende for <b>Samfundet</b> . Alle epidemiske og smitsomme Sygdomme har altsaa hidtil været behandlet af de samme Funktionærer, som har med Fattigpleien forøvrigt at gjøre , — tildels med extraordinær Hjælp rigtignok , — og for Fattigvæsenets Regning . Kun under store Epidemier , som Kolera , og leilighedsvis under andre Epidemier er der gjort Undtagelser herfrå . Blandt disse Sygdomme , Fattigvæsenet har taget sig af . er ogsaa en , som mindst af alle egentlig synes at burde indgaa under Fattigadministrationens Omraade , det er de Foranstaltninger , som træffes mod den veneriske Sygdom . Endel af Udgifterne ved disse bæres af Bykassen , nemlig forsaavidt Visitationen af de prostituerede <b>Kvinder</b> angaar ; men det er ogsaa Alt . Forpleiningen paa Sygehus af dem , som tvungent eller frivilligt indlægges der , foregaar for Fattigvæsenets Regning . Selve Administrationen af Prostitutionsvæsenet henligger vistnok under Sundhedskommissionen og det almindelige Politi ; men der benyttes dog dertil ogsaa Funktionærer , som lønnes af Fattigvæsenet , nemlig Assistenterne ved Politikammeret for Fattigvæsenets Anliggender , saaat Fattigvæsenet ogsaa er blandet ind i denne Sag . Desuden har Fattigvæsenet overtaget Omsorgen for de Sindssyge , saaat det ikke alene betaler for deres Forpleining , men ogsaa seiv holder et Sindssygeasyl, som administreres af Fattigkommissionen . Det er tydeligt , at paa denne Maade overføres der paa Fattigkassen en stor Del Udgifter , som i Landdistrikterne ikke bæres af vedkommende Kommune , men af Amtskomnmnen, ialfald for en større Del . Alle de Sindssyge , som er i den Tilstand , at de tiltrænger en fra den sædvanlige Fattigforsørgelse forskjellig Behandling , underholdes nemlig efter Loven af Amtskommunen for 4 / s Dele ; her underholdes disse helt ud af Fattigkassen . Det samme er Tilfældet med de epidemiske Sygdomme i Landdistrikterne , naar Sygdommen er erklæret for Epidemi. De Ting , som her er taget under Fattigvæsenet , gjør altsaa , at det Hele bliver et temmelig vidtløftigt Administrationsfeldt . Forpleiningen af de <b>fattige</b> Syge foregaar dels i <b>Hjemmet</b> , dels i <b>private</b> Familier , hvor de Syge udsættes , dels i Sygehusene og for en ganske ringe Del i Arbeidsanstalten . De Sindssyges Forpleining sker enten i <b>Hjemmet</b> eller ved Udsættelse i <b>privat</b> Forpleining eller i Kommunens Sindssygeasyl.

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081903015'target=_>Sundt, Eilert, 1859, Fortsat Beretning om Fantefolket</a><h4>822</h4>ning i Familiens Frihed , havde han gjentaget sit Paaleg til Lensmanden om specielt Opsyn med dens Forhold og anmodet ham om at afgive terminvis Indberetning . Mundtlig tilholdt Fogden Christian Fredriksen at stille sig ved Hunden som overflodig og upassende for en Mand i hans Stilling ; ligesaa foreholdt Fogden ham det Unodvendige . i at tåge Gever med sig paa sine Vandringer ; endelig gjorde Fogden ham bekjendt med , at Fattigkommissionen onstede , at hans <b>Kvinde</b> og Datter og de smaa Born skulde vcere hjemme . At gjore noget Videre ved dette , ester Fattigkommissionens Forlangende , fandt Fogden upassende og for en Del stridende mod den Frihed , som Loven hjemler Enhver, naar man intet Ondt foretager .  Politiet , saaledes fortsetter Fogden , vilde paadrage sig Daddel , naar det udenfor Lovens Grcendser sogte at indskrenke Folks Frihed , og med Hensyn til disse Mennesker i Scerdeleshed vilde man " handle uklogt og arbeide mod Hensigten , om man ved altfor strcenge Forholdsregler drev dem tilbage paa Fantestien . Selv om de imellem vandre ud af det Prestegjeld , hvor de nu have fast Bopel , er dette Noget , som man ikke vel kan eller bor forbyde dem . Gjore de noget Galt , vil det altid blive spurgt og anmeldt ; thi disse Folk , som ikke yndes , og som sees paa med stjceve Dine af Alle , staa under hele Almuens specielle Opsyn ; man tror dem onde og vil heller ikke opdage andet end Ondt hos dem . Agtelse og Tillid er ikke nogen let Sag for dem at erhverve , og deres Sammensmeltning med den ovrige Almue vil ikke lettelig eller snart kunne paaregnes . , Deres gamle Vandre- eller Fanteliv har vel medfsrt Vanskeligheder og Bcsverligheder for dem ; men deres Civilisation eller Optagelse i det velordnede borgerlige <b>Samfund</b> er heller ikke forbunden med synderlig Opmuntring eller Glede . De trenge derfor hoilig til at tages i Forsvar og holdes under Armene . " Om den tilbag ev endte C ar o line yttrer Fogden :  Hun er et <b>moralsk</b> fordervet Menneske . Hendes Optagelse paany i denne Kreds er derfor saare übehagelig baade for Prest og Almue . Ogsaa for mig var hendes Tilbagckomst mindre end behagelig . Men hun og hendes 2 smaa Born ere Mennesker , som man ikke kan lade sulte ihjel . " — Sluttelig anbefaler Fogden Christian Fredriksen og trengende <b>Familie</b> til fremdeles at erholde Understottelse af Statskassen .  Kan det , tilfoier han , ske under et bedre Politi-Opsyn end her , onsker Ingen dette heller end jeg , og jeg vover endog at andrage paa , at Saadant foranstaltes , samt at ' ialfald Caroline Christiansdatter anbringes i en Rednings- Anstalt , og at hendes to Born saavelsom det 3 die Barnebarn , som har veret hos de Gamle , saasnart Omstendighederne tillade det , vorde anbragte i Bsrne-Asyler eller paa anden Maade for

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012071305102'target=_>, 1870, Nordiske Digtere i vort Aarhundrede</a><h4>8371</h4>Rahbek og hans geniale Samtidige , Baggesen og Flere . Men især har jeg i sildigere , mindre lyse Dage fundet et Fristed mod alle Sorger i det for saa mange uforglemmelige Bakkehuus . I dette Gjæstfrihedens <b>Hjem</b> bleve Menneskene veiede efter ganske andre Regler end i den øvrige Verden . Det gik til som i Himmerige , hvor Rang og Rigdom Intet gjelde . Den <b>Fattige</b> , Den , som Verden oversaae , men som besad noget Kunstnerværd , nogen Fortjeneste , var sikker paa her at finde et Asyl , hvorhen han kunde tye i mørke Timer og hente Trøst , vis paa at blive modtagen med et varmt Haandtryk og et mildt Ansigt af den altid ungdommelige Knud Lyne , og at blive forstaaet og oplivet for mange Dage i Samtale med hans aandrige <b>Hustru</b> . Hvormange samlede sig ikke der , af Alt hvad <b>Nationen</b> i to Generationer har at opvise af Herligt og Udmærket ! I denne Kreds , under dette lave Tag , hvor Meget er der ikke grundlagt, der siden har spiret og baaret Frugt , ligesom Havens Træer og Blomster , plantede og pleiede af Cammas Haand ! Havde Soerates levet , han havde besøgt dette Huus med Plato og Alcibiades . Stege Jupiter og Mercur igjen ned til Jorden , de vilde besøge dette » » Tempel , som Baucis og Philemon dannede af en Hytte . « « Naar jeg endnu for faa Aar siden , ligesom i Aften , vandrede , til Vestergade i Kjøbenhavn , hvor glædede jeg mig da , hvergang jeg i den tidlige Morgenstund nærmede mig dette kjære Tag , og saae Røgen opstige derfrå ! Naar jeg gik hjem igjen , tilbragte jeg først Aftenen der , og hentede Styrke og Munterhed til min natlige Vandring . Nu seer jeg bedrøvet ned ad Veien , under hvis Træer jeg og saa mangen Anden tilforn ilede med Glæde . Jeg hører , at der findes Dem , som dadle Støtten , som staaer i Alleen . Jeg siger , Støtten er god , og Indskriften herlig . Hvo der har gjort den , veed jeg ikke ; men jeg ærer høilig dens Forfatter ; han har følt som jeg , hvad der ligger i de faa Ord : » » Her boede Knud Lyne Rahbek og hans elskelige <b>Hustru</b> . « « Her boede de ; de boe her ikke mere . Vend om , stakkels Pilgrim ! Du finder ikke mere den gamle trofaste Sanger og hans elskelige <b>Hustru</b> . Ja elskelig tilvisse ! » » 0 Margarita , Perle reen og klar ! « «

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112301019'target=_>, 1859, Norsk lovsamling for Aarene 1667 til 1858</a><h4>955</h4>til disse offentlige Gjordemoder - Indretninger . 18 ) Saa stal og enhver Kjsb » stad og ethvert £ anbbiflrict forsyne den ansettende © jorbemober med de Redstaber , som ansees fornodnc til hendes Bestilling . Disse anffaffeé førfte Gang paa Kjsbstadens etter Districtets Bekostning , og overleveres Gjordemoderen ved Syn , samt tilbakeleveres paa lige Maade . 20 ) Naar Veiens Lengde etter Omstcendighederne krever , at Gjordemoderen maa betjene sig af Vogn , da bor de , som begjcere hendes Hjcelv , enten selv lade hende afhente og hjemfsre , eller og godtgjere hende de Omkostninger, hun paa nodvendig Befordring maa anvende . 21 ) Ligeledcs al Gjordemsdrene i Landdistricterne , naar de kaldes til at betjene <b>Fattige</b> , eller Huusmends og Indersters Hustruer , der ikke selv have Heste og Vogn , nyde fri Befordring frem og tilbage . Denne Befordring paaligger bet Districtets Gaardmend at afgive ester Omgang , og paa Tilsigelse af Sognefogden eller Oldermanden ' ) . 22 ) Derimod flat samtlige Gjordemsdre , som efter det Foregaaende ere ansatte med fast Lsn , uden Betaling betjene alle de <b>Fruentimmer</b> , som med Attest fra rette Vedkommende godt- » gjøre , at de nyde Underststtelse af Fattigvesenet . 23 ) Under disse Vilkaar stal t ethvert Gjordemoder-District snarest muNgt anscettes en oplert Gjordemoder , som bringes i Forslag af Stedets Dvrighed efter Overlceg meb Districtets Lcege , og beflikkes af Oversvrigbeden . 24 ) Naar der iet District opholde sig flere eraminerede Gjordemsdre , og Vacance indtrceffer blandt de Gjordemsdre , ber have Lsn , da bør Dvrigbeden , » eb at befcette den ledige Plads , fornemmelig tåge Hensyn til ben ulsnnede Gjorbemoder sammesteds , ber bar udmerket sig ved Flid og Duelighed ; uden at ben tcengere eller kortere Tid , i hvilken hun har veret ® jorbemober , stal komme i Betragtning. 25 ) I Almindelighed maa Ingen bestikkes til Gjordemoder under ovennevnte Vilkaar , medmindre hun er forsynet meb Testimonium om sin Dueligbed fra den Kgl . Gjordemoder-Commission ; faa bør hun beéuben have forbeetag * tigt Vidncsbyrd om <b>Sedelighed</b> fra hendes Sjelesorger . 26 ) Skulde der ved denne Indretnings Iverksettelse ikke sindes et tilstrekkeligt Antal af ssige ordentlig oplerte Gjordemsdre . ba vil Kongen , paa bet at intet Gjordemoder-District stal savne Fsdselsbjelpcre, have Oversviighederne bemyndiget til t ssige Tilfelde at give saadanne <b>Fruentimmere</b> Tilladelse til at practisere som Gjordemsdre , der enten ere blevne oplerte af © ttftø- og <b>Land</b> - Physicis , og af disse have erholdt Vidnesbyid om deres T ^ upTtg ^ pb i psspv og , efter Dvrighedens Foranstaltning , af Stifts- eller Land-Physico , i Overveielse af Distriktslegen eller anbcn lcegekyndig Mand , ere blevne prsvede og befunbne og have den Kundstab og gcerbfgtjeb , at det , uden at befrygte nogen Fare , kan tillades bem at betjene Sarfelquinber . 27 ) Disse af Stifts - eller Landphysicis saaledes oplerte etter prø » ebe <b>Fruentimmere</b> , hvilke af Mangel paa ordentlig eraminerede Gjordemsdre ansettes i Districterne . stal ikke tillegges den for Districts-Gjordemsdre i de foregaaende §§ bestemte £ øn etter de øortge faste Emolumenter , mebminbre de , af Mangel paa duelige Subjecter i selve Districtet , kaldes bertit fra et andet Di- » strict , og altsaa maa stytte fra deres Hjemstavn . Derimot » ssal de vere berettigede til at nyde ben i §§ 19 , 20 og 21 bestemte Betaling og frt Befordring , samt , naar de betjene de i § 22 nevnte <b>Fattige</b> , da tillige Betaling af vedkommende Fattigvesen ; ligesom det ogfaa ssal vere bem uformeent at vedblive deres Praxis , om endog en ordentlig examineret Gjordemoder bliver ansat i Districtet , hvilket bsr stee . saasnart Omstendighederne tillade det . 29 ) Naar der til et Gjordemoder-District ikke kan erholdes nogen ordentlig oplert Gjordemoder , ba stal Stedets Dvrighed derom gjøre Indberetning til Oversvrigheden , samt bringe eet eller flere Subjecter , helst af Di » strictets egne Beboere , i Forslag til at opleres t © jorbemober » ibenff aben , for ftbeit at ansettes som Gjordemoder i Districtet . 30 ) Det <b>Fruentimmer</b> , ber bringes i Forslag til Oplerelse som Gjordemoder , maa ikke vere under 20 Aar og iffe over 30 Nar gammel ; hun bør have Beviis fra sin Sjelesorger om benbeé © cebetigbeb , og Attest fra en Lege om hendes Physiste Beqvemhed ; hvorved tittige maa iagttages , at hun ikke bar nogen Legemsbrek eller Mangel , som kunde opvekke Skrek eller Afsiye hos den Fsdende ; ogsaa stal , under lige Beqvemhed til at bringes i Forslag , den nydeFortrin , som er gift og selv har føbt Søm ; tigefom og den , " ber kan skrive , bør foretrekkes Andre , der ikke forene denne Ferdighed med de svrige udfordrende Egenstaber. 31 ) Dersom der iblandt de <b>Fruentimmere</b> . som saaledes bringes i Forslag , stulve findes Nogen , der besad alle fornodne Egenstaber , men ikke havde Evne til selv at bekoste sin Oplerelse . da stal Oversvrigheden . naar den sinder saadant efter Omstendighedcrne nsdvendigt , vere bemyndiget til at foranstalte bende paa bet Almindeliges Bekostning underviist ved Lereanstalten for © jorbemøbre . 32 ) Omkostningerne paa en faaban gærltngg Dp ^ otb og UnberoiiSntng i Stiftelsen , samt paa hendes

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092203005'target=_>, 1861, Almindelig Norsk Lovsamling</a><h4>7136</h4>fulbbyrdet , naar der nemlig er Grund til at antage , at Vedkommende derefter agte at indtrcede i oldentllgt Wgteskab . — Ved denne Hjelp til Undervisning og Konsirmatlonsforberedelsen, som i Regelen vil medtage noget lengere Tid , ber det tillige håves for Vie , at Vedkommende funne vennes til stadigt Arbeide eller vel endog oplcrres l en eller anden nyttig Haandtering ; c ) at forhjelpe eldre arbeidsføre Personer , med eller uden <b>Familie</b> , til at erholde et fast <b>Hjem</b> og fomme ind paa en ordentlig Næringsvei . De Tilstud , der en Gang for alle eller aarligen hertil ville udkrceves , ber af Fattigkommisfionerne eller andre Vedkommende , saavidt muligt , disponeres til de Angjeldenbes Vedste uden at leveres dem selv i Hende . Det er en Selufelge , at llgesom saadan Understettelse fra Begyndelsen vilde vcere tllstaaet i Haab om , at Angjeldende ved Flid og Orden lidt efter lidt kunne komme i saadan Stilling , at de kunde erncere sig selv paa ordentlig Maade og som bosatte Folk , saa maatte ogsaa Understøttelsen , forsaavidt den er tilstaaet aarlig , ophere , naar dette Maal er naaet . Forsaavidt disse Folk have Bern , vil det serligt verre at afgjere , om disse ber forserges saaledes , som under Litr . l > antydet , eller overlades til Foreldrenes Forsorg ; 6 ) at yde sygelige , vanfere og alderssvage Folk , som Intet eller kun Lidet kunne erhverve , et fast <b>Hjem</b> og fornedent Underhold i Lighed med andre <b>Fattige</b> , og selvfølgelig under Betingelse af , at de leve roligt og semmeligt . 3 ) l > ) Understettelsen vil i Regelen blive at anvende gjennem Fattlgkommissionerne og efter disses Forslag . Hvorvidt Formaalet sial naaes og det for vedkommende selv saa ulykkelige og for <b>Samfundet</b> saa fordervelige og beskjemmende Omstreiferucrscn stal vorde medarbeidet , vil derfor vesentlig bero paa Fattigkommissionernes Iver og Udholdenhed ; b ) Naar en Fattigkommission har antaget sig en eller siere af omhandlede Personer med den Hensigt , at sege noget af de nevnte Formaal for dem opnaaet , — naar Fattigkommissionen har antagelig Vished for , at Angjeldende ikke have Hjemstavn i noget af Nigets Fattigdistrikter , og naar den efter sit Kjendstab til Personerne og Omstendighederne forevrigt har sundet Anledning til at fremsette Forsiag om den hensigtsmcessigste Understettelse for dem , ber den gjennem vedkommende Amt og Stiftsdlrektion til Departementet indsende dette Forslag , ledsaget , furuden med Overslag over de sandsynlige Udgifter og Tiden for disses Varighet , , ogsaa med saadanne Forklaringer , hvorefter det kan sijennes , at ' Angjeldende rettelig ere til det omhandlede Slags Personer at henregne og at de ere hjemstavnslose . I saadant Tilselde vil Erstatning for Udgifterne ved den midlertidige Forsørgelse vorde udredet af de for dette Anliggende bevilgede Midler , og efter Omstendighederne vil ogsaa den yderligere Underftettelse , som hint Forslag maatte gaa ud paa , vorde tilstaaet ! « ) Dersom en Fattigkommission i Felge Fattiglovens Medfer er kommen til at antage sig nogen i Distriktet ikke hjemmehørende Person til midlertidig Forsørgelse , og da ved de derom anstillede Undersegelser erholder antagelig Vlshed for , at samme Person er hjemstavnsles og derhos efter Omstendighederne skjennes at henhere til det her omhandlede Slags Personer , saa ber Fattigkommissionen ogsaa i dette Tilfelde , nest efter at yde den fornsdne midlertidige Understettelse , tillige bestrebe sig for at finde Anledning til at yde en saadan videre gaaende Hjelp oq Bistand , som ved de heromhandlede bevilgede Midler er tilssgtet . Finder Fattigkommissionen da Anledning til at virke noget i dette Biemed , saa behandles Sagen ganske saaledes , som under Litr . b er sagt . Finder Fattigkommissionen derimod , at der ikke er Anledning til ot foretage noget Vderligere , efterat de Omstcendigheder ere opherte , som foranledigede den midlertidige Understøttelse , saa bliver ogsaa derom snarest muligt , helst inden angjeldende Personers Bortreise fra Distriktet , gjennem vedkommende Amt og Stiftsdirektion til Departementet at afgive Beretning , ledsaget med fornsden Forklaring om de Grunde , der have ledet Fattigkommissionen ; < 1 ) dersom nogen i et bestemt Fattigvesensdistrikt hjemstavnsberettiget Person alligevel i Folge sine Livsforholde forevriat kan ansees at henhere til det her omhandlede Slags Personer , og dersom Distriktets Fattigkommission efter en omhyggelig Undersegelse finder , at der ved en noget videre gaaende Understettelse end den almindelige Fattigforsergelse kan vcrre Haab om at faa samme Person bragt ind i en ordentlig Neringsve ! og fast Livsstilling , saa vil der efter Omstendighederne kunne tilstaaes noget Bidrag til Bestridelse af de foregede Omkostninger , Som Erempel paa Tilfelde , hvor et saadant Bidrcig vil kunne ventes , anferes , at en i Distriktet hjemstavnsberettiget Mandsverson har levet i Konkubinat med en ikke i Distriktet hjemstavnsberettiget <b>Kvinde</b> , og at de ved den paapegedr Understettelse kunne haabes at ville indtrede i ordentligt Mgtesiab og tåge fast Ophold i Distriktet . 4 ) Personer af det her omhandlede Slags staa , saa lenge de nyde Understettelse , i samme Forhold til Fattigkommissionerne som andre <b>Fattige</b> , der ere under deres Forsorgelse . Paa Grund af det Seregne ved disse Personers Forholde ber dog Fattigkommlssionerne efter Omstendlghederne end videre ved seregne Foranstaltninger fere et neiagtigt Tilsyn med dem og vaage over , at det gode Formaal , man

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006082800008'target=_>Sundt, Eilert, 1855, Om Giftermaal i Norge</a><h4>79</h4>5. Tet er overmaade vigtigt , at Meningerne om denne Sag bringes til et fuldstcrndigt Opgjor . Skulde det vcrre saa , at det ikke alene ofte var Tilfcrldet , men at det endogsaa blev mere og mere almindrligt , at raa og <b>fattige</b> Folk giftede sig i ung og utidig Alder , at der vistes en sorgelig Mangel paa besindig Omtanke ved et Skridt , som er af saa stor Viatighed baade for den Enkeltes og for Samfundets Vel . ' at stifte <b>Familie</b> — skulde det vcrre saa . da maatte det endelig vides med uimodfigelig Vishrd , paa det at alle Samfundets sunde og gode Krcrfter kunde forene sig om at overvinde et saadant Grund-Onde . Eller skulde det vcrre saa vel , at hint Onde ikke greb videre om sig , men kanske heller var i Aftagende , saa burde dette ogsaa vides . Den Mening , at <b>Samfundet</b> trues med Fare formedelst den store Hobs Raahed , er vel stikket til at fremkalde Anstrengelser fra de hoiere staaende Klassers Side ; men den er ogsaa stikket til at scrtte ondt Blod , til at avle Bitterhed og Misnoie paa begge Sider . Har man derimot » den Mening , at Samfundslivet heller arter sig til det Bedre , tror man at finde , at den paa Samfundets Holder stigende Oplysning og <b>Scrdelighed</b> virkelig lidt efter lidt treenger igjennem og

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010020412003'target=_>Sundt, Eilert, 1850, Beretning om Fante- eller Landstrygerfolket i Norge</a><h4>681</h4>kommen over Nyggcu af den Aas , som hcevcr sig mellem Mjosen og Randsfjordm . Mod Nord tabte Blikket sig i et dybt Perspektiv af vakkre Dalfsrcr , som udmunde sig mod dm nordlige Vig af Randsfjorden , og lige under den Hside , hvor jeg befandt mig , viste denne stjonne So en Del af sit klare Speil , ngt infattct af lyse Enge og gronne Lier og derover mcegtige Fjeldrader , som hevede sig over hinanden i det blaanende Fjerne og skjermede for den lune , af Sommerdufte fyldte Dal . Men det er ofte Tilfceldet i vort <b>Land</b> , at Naturen aabcnbarer sin mest storartede Skjonhcd netop omkring de <b>fattige</b> Husmandsstucr og Nybyggerpladse , hvor de Seende ikke se og de Horende ikke hore . Netop der , hvor jeg uvilkaarlig standscde min Hest for at kvcege mit Sind ved det vakkre Syn , laa en Nybyggerplads . Hvor den lille Vygagcr ved Siden af var vantreven ! hvor de halvnogne Vom , som grov i Sandet udenfor Doren , vare blege og forknytte ! Det ikke gamle Hus syntes allerede faldefcerdigt ; i Vinduerne havde Filler og Papirlapper allerede indtagct nogle af de faa Ruders Plads . Alting bar Preg af overvettes Armod , og Intet vidncde om Flid og Ordenssands . Jeg syntes aldrig at have feet en saa ussel Volig ; den Tanke stog mig , at denne Hyttes Beboere , overveldede og slovede af Sorger for dette Livs Nering , neppe havde bevaret Sands tilovers til at glede sig ved Synet af det deilige Landstab , som udbredte sine Skjonhcder for deres Vine , og jeg steg af Kanolen og gik ind med Tanken om her at sinde et sjelden ssrgeligt Villede af Aandsfattigdom og Slovhed forenet med den ydre Armod . Men nctop her sik jeg opfanget et ganske slaaende Erempel paa , hvor stor Afstaud der kan vcere mellem selv en saadan lavt staaende <b>Familie</b> af det borgerlige <b>Samfund</b> og de lost omstreifende Fantefolk. De forste Meddelelser af det middelaldrende Mgtepar , jeg fandt i Hytten , vare ingenlunde fordelagtige . Det blev mig klart , hvad jeg ogsaa siden af Andre fik Stadfestelse paa , at Ustadighcd og andre Udyder for en stor Del vare Skyld i deres nuvcercnde Elendighed . Nu erncercde de sig mest derved , at Mandcn bandt Kurve , som <b>Konen</b> gik omkring i Vygderne

<h4>1434</h4>Musikdyrker , bereiste nylig paa offentlig Bekostning nogle af vore Fjeldbygdcr for at opfange de eiendommelige Psalmemelodier og verdslige Sange og Slaatter , som der have hjemme . Hvorledes mener man nu han bar sig ad dermed ? Havdc han uden videre som Fremmed anmodet Vsnderfolkene om at synge og spille for ham , skulde de vistnok kun seet paa ham med Forundring eller vendt sig sra ham med Mistro . Derfor sang og spillede han selv for dem forst , og nu flokkede de sig om haln , Mcend og <b>Kvinder</b> , Gamle og Unge , og lyttcde med Henrykkelse; naar saa han holdt inde , saa var det , som om besicegtede Strenge vare slagne an i deres Vryst ; thi nu sang og spillcde de af Hjertens Lyst , medens han horte ester . — Saalcdes maa den , som i et eller andet Stykke snster at hore Almuesmandens Tanker , vide at scette sig i et livligt Verelforhold til ham ; han maa bcere med sig visse Anskuelser og Interesser og meddele dem til Andre , for saaledes at faa se disse meddelsommc igjen . Om Nogcn til Erempel vil soge at gjore sig fortrolig med de religwse Forholde i en Bygd , saa maa han selv vcere religiossindet og vel forfaren i Guds Ord og saa i Samtale med dem , han moder paa sin Vei , vise Lyst og Gave til at underholde sig om alvorlige Ting , ja til at formane og opbygge , og jo mere oprigtig og nidkjcer han lelv er , desto mere ville Folkets Hjerters Tanker aabenbare sig for ham . — Onsker Nogen at skaffe sig Kundssab om , hvordan Fattigvesenet har udviklet sig paa et Sted , hvordan Forholdet er mellcm <b>Fattige</b> og Rige , saa maa han forud have Skjon paa de vrange Vcgreber og Forviklinger , som i dette Forhold let kunne opstaa , og vcere besjelet af Lyst til at se alle Misligheder rettede , og han vil mode Fortrolighed i samme Grad , som han ved sin Tale allerede viser sig at vcere fortrolig med Alt . — Der er , for endnu at anfore et Erempel , en Mislighed, som i mange , ja i de fleste Egne i vort <b>Land</b> klceber ved den lavere Almues huslige Liv , nemlig en hoi Grad af Urenlighed, som navnlig ogsaa af den Grund er saa fordcrrvelig for det <b>moralske</b> Liv , at den maaske mere end noget Andet vedligeholder en bred Klsft mellem de hoicre og lavere Klasser ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013072208224'target=_>, 1880, Vidnesbyrd fra den norske Kirke</a><h4>239</h4>Kristi gode Evangelium ikke er blevet troet ! Kristi Rost er Freds Rost , Kristi Sind er Selvfornegtelsens Aand , Kristi Gave er Kjcerlighedens Hjertelag , og rundt i Kristenheden hores i de storre <b>Samfund</b> Krig og Rygte om Krig , i de mindre <b>Samfund</b> Uenighed og Splid , at den Ene misunder den Anden , at den Ene forfordeler den Anden , at Fordragelighed kun kan bevares under Skilsmisse . Kristi Sind vender Blikket indad og boier Hjertet til Idmyghed , og i hvad der kaldes kristne <b>Samfund</b> , hsres Dommen over Andre at hceve sin Rost , medens Sporene af , at den Samme dommer sig selv , ere skrevne i Flyvesand ; Hovmodens Pande lofter sig frcekt , og saa Mangen begjcerer aabenbarlig at vcere den Forste for at herske efter sit Hoved . Kristi Sind gjor os til Udlændinge i Verden og stab er Lcengfelen , den stille , men daadkraftige Lcengfel efter <b>Hjemmet</b> heroventil, og se , midt i Kristenheden fcerdes Folkene , som om her var deres blivende Sted ; her er deres Liggendefæ , her ville ogfaa deres Hjerter vcere ; i det , fom er idag og imorgen kastes i Ovnen , deri soger man sin Gloede og sit Liv ; dets Fylde er Herlighed , dets Savn er Kval . Kristi Sind er Alvorets Nand ; Kristi Tro staber Mcend , som vide , hvad de ville i Herren , <b>Kvinder</b> , som vide med Hjertet , hvad de elste i Herren , og midt i Kristenheden tumle sig de Mange , hvis Liv er mcerket med den flagrende Forfængelighed , hvorpaa de have kastet , hvad af en stakkels Kjcerlighed det <b>fattige</b> Hjerte eier , de Mange , hvis Idrcet er den tomme Forfængelighed , hvad enten den er Arbeidet kun for det , som horer denne Verden til , eller Glceden kun i det , som er af denne Verden , ak , altfor ofte idel tom Forfængelighed, selv om den laaner en Form af et aandeligt Vcefen , der altfor fnart aabenbarer sin Formhulhed ved sin Halvhed og sin Flygtighed . Kristus Jesus gjor Menneskenes Born velbehagelige for Ham , hvem Ingen stal fe uden Hellighed , renfer deres Hjerter og bliver selv deres Aands Liv og Tragten ; men i Kristenheden hores ikke alene om enkelte Misgjerningsmcend , men Manges daglige Liv gaar gjennem Overtrædelser , om ikke af Menneskers , saa dog af Guds Lov . De tjene uden Frygt sine egne Begjceringer og have Mod og Hjerte til at bruge og forfvare , hvad de dog felv vide at vcere mod Aanden . Sandelig , det er mere end forunderligt , at Guds store og gode Gjerning og Guds Vudstab om sin Gjerning har indtil denne Stund udrettet saa Lidet i Verden , faa Lidet mellem os , saa lidet er blevet modt og brugt med Tak og Tro . Det er mere end forunderligt , at faa er steet , og at Gud dog bliver ved med at forkynde sit Vudstab , at tilbyde os sin Gjerning ; vor forunderlige Gud kalder sig jo ogfaa selv Taalmodighedens Gud .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081403021'target=_>, 1863, For Hjemmet</a><h4>2729</h4>Maade . hvorpaa han optog Shelleys Plan , gik imidlertid i en ganske anden Retning , end denne havde ventet , thi istedetfor at opmuntre til Handling , raadede han indstcrndigen til Forsigtighed og formanede ham til ikke at lade sine Tanker fremtlcrde i Skrift og Tale . Men saadanne Raad vare spildte paa Shelley . Han vilde ikle opgive Irlands Gjenfsdelse . Han drog til Dublin og begyndte at gaac frem der paa en Maade , som fremkaldte endnu stcerkere Advarster fra Godwin end forhen . Han fandt i Dublin menneskelig Elendighed og Usselbcd i en Grad . som han aldrig fsr havde scet ; de <b>Fattige</b> vare pakkede sammen i trange Gader lig en Masse af belivet Smuds ; Psbelen var saa degraderet og dyrist , at han troede Astersen stod hsiere end den i intellectuel Henseende . Dette var det haabefulde Auditorium , hvortil han henvendte Flyvcskrifter angaaende deres Nations Stilling . Han forestog at hcrve Dubliner-Psbelen fra den vanancnde Lethargi , hvori den var nedsunket , ved at danne Selskaber til Afhjelpelse af Elcndigheden . Det var i Sandhed alene Eiistencen af saadan Elendighcd , som synes at have givet ham noget Haab om den Gjenfsdelse , han vilde bevirke . Han havde lånt af Godwins Skrifter , at en hsi <b>Moral</b> maa ringeagte det <b>private</b> Livs Bagateller og låne Mennefferne at betragte sig i det Forhold , bvori de staae til det hele mcnncstelige <b>Samfund</b> . Det forekom ham derfor at virre en heldig Omstamdighed , at dette fornedrede Folks godgjørende Tilbsieligheder til en vis Grad bleve vakte og in » dividuellc Interesser bragte til at antage en almindeligcre Cha » racteer ved Forbindelsen med Lovbudet om Katholikerncs Udelukkelse og den Undertrykkelse , som var en Folge af Unionsacten . Dette var en Krisis . som ikke burde gaa hen « benyttet for hans 2 ie « med. og den Maade hvorpaa den skulde benyttes , var Indstiftelsen af et Selskab med det Oiemed , at gjengive Irland den Lykke , det besad fsr Unionen . Men Godwin erindrede lacobincrne

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111401060'target=_>, 1877, Illustreret Verdenshistorie</a><h4>1545</h4>Det er tydeligt , at de sornemme Romere ved Deltagelse i Forretningslivet og ved den Anledning til at samle Penge , som Krigene og Provindsbestyrelsen tilbpo , maatte blive meget formuende . Man ser ogsaa , at Maaleftokken for , hvad der ansaaes for Rigdom , var meget HK . Saaledes gjaldt Lucius Paullus ikke for rig , skjpnt han eiede omtrent 270,000 Kroner , og den Medgift , fom enhver af den « eldre Scipios Dptre sik , noget over 240,000 Kroner , ansaaes ikke for mere end rimeligt for en <b>Pige</b> af saa fornem <b>Familie</b> . Men ved Siden af de Rige , stod en Befolkning , der levedo for Dagen , og alt mere og mere vcennede sig til at fordre Almisser af Staten og af <b>Private</b> . Klpften mellem Samfundets forskjellige Klasser udvidedes mere og mere , iscer da de fornemme og rige i Modscetning til de gamle Dages Opfatning ogsaa ssgte at bcere denne Forskjel til Skue i sin ydre Optrceden . I Hceren tjente nu ikke lcengere de afholdende og hcerdede Krigere , som , idet de kjcempede for <b>Fcedreland</b> og Guder , for <b>Hustru</b> og Bprn , Hus og <b>Hjem</b> , havde vceret den gamle Tids Styrke , men i deres Sted var traadt Personer , fordcervede af Ippighed og kun begjcerlige efter at gjsre Bytte , og ved ' Triumferne kapvedes Feltherrerne om , hvem der kunde uddele flest Penge til sine Soldater , der hver fik lige indtil 500 As , medens Centurioner og Ryttere fik det dobbelte og tredobbelte . Sin Stilling opretholdt Nobiliteten ved glimrende Fester , dels de egentlige Lege , som Prcetorer og TEdiler havde at foranstalte , dels overordentlige Skuespil , arrangerede for at opfylde Lpfter , givne i Krigen . Slige Fester vare meget kostbare ( Gladiatorlegene kostede 720,000 Seftertier eller omkring 136,000 Kroner ) , men de vare nødvendige Ofre for den , der vilde frem paa Embedsbanen og altsaa maatte beile til Mcengdens Gunst . De glimrende Festers og Triumfernes Pragt blcendede Tilskuerne , men bag alt dette laa en Fattigdom og en <b>moralsk</b> Fordcervelse , der maatte fylde enhver dybere tcenkende og endnu ikke af Magtens Spdme ganske bedcervet Borger med dyd Bekymring for Fremtiden .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008090903001'target=_>Sundt, Eilert, 1852, Beretning om Fante- eller Landstrygerfolket i Norge</a><h4>661</h4>kommen over Ryggen af den Aas , som hever sig mellem Mjesen og Randsfjorden . Mod Nord tabte Blikket sig i et dybt Perspektiv af vaktre Dalforer , som udmunde sig mod den nordlige Vig af Randsfjorden , eg lige under den Heide , hvor jeg befandt mig , viste denne stjonne So en Del af sit klare Speil , rigt indfattet af lyse Enge og grenne Lier og derover megtige Fjeldrader , som hevede sig over hinanden i det blaanende Fjerne og skjermede for den lune , af Sommerdufte fyldte Dal . Men det er ofte Tilfeldct i vort <b>Land</b> , at Naturen aabenbarer sin mest storartcoe Stjonhed netop omkring de <b>fattige</b> Husmandsstuer og Nybyggerpladse , hvor de Seende ikke se og de Horende ikke hore . Netop der , hvor jeg uvilkaarlig standsede min Hest for at kvege mit Sind ved det vakkre Syn , laa en Nybyggervlads . Hvor den lille Bygager ved Siden af var vcmtreven ! hvor de halvnsgne Bern , som grov i Sandet udenfor Doren , vare blege og forknytte ! Det ikke gamle Hus syntes allerede faldeferdigt ; i Vinduerne havde Filler og Papirlapper allerede indtaget nogle af de faa Ruders Plads . Alting bar Preg af overvettes Armod , og Intet vidnede om Flid og Ordenssands . Jeg syntes aldrig at have seet en saa ussel Bolig ; den Tanke slog mig , at denne Hyttes Beboere , overvelocde og slevede af Sorger for dette Livs Nering , neppe havde bevaret Sands tilovers til at glede sig ved Synet af det deilige Landstab , som udbredte sine Skjonheder for deres Dine , og jeg steg af Kanolen og gik ind med Tanken om her at sinde et sjeldent sorgeligt Billede af Aandsfattigdom og Slevhed forenet med den ydre Armod . Men netop her sik jeg opfanget et ganste staaende Erempel paa , hvor stor Afstand der kan vere mellem selv en saadan lavt staaende <b>Familie</b> af det borgerlige <b>Samfund</b> og de lost omstreifende Fantefolk. De forste Meddelelser af det middelaldrende Mgtepar , jeg fandt i Hytten , vare ingenlunde fordelagtige . Det blev mig klart , hvad jeg ogsaa siden af Andre fik Stadfestelse paa , at Ustadighet ) og andre Udyder for en stor Del vare Skyld i deres nuverende Elendighet » . Nu crnerede de sig mest derved , at Manden bandt Kurve , som <b>Konen</b> gik omkring i Bygderne

<h4>1438</h4>Musikdyrker , bereiste nylig paa offentlig Bekostning nogle af vore Fjeldbygder for at opfange de eiendommelige Psalmcmelodier og verdslige Sange og Slaatter , som dcr have hjemme . Hvorledes mener man nu han bar sig ad dermed ? Havde han uden videre som Fremmed anmodet Bonderfolkenc om at synge og spille for ham , stnlde de vistnok kun seet paa ham med Forundring eller vendt sig fra ham med Mistro . Derfor sang og spillede han selv for dem forst , og nu flokkede de sig om ham , Mend og <b>Kvinder</b> , Gamle og Unge , og lyttede med Henrykkelse; nåar saa han holdt inde , saa var det , som om bestegtcde Strenge vare slagne an i deres Bryst ; thi nu sang og spillede de af Hjertens Lyst , medens han horte efter . — Saaledes maa den , som i et eller andet Stykke onster at hore Almuesmandcns Tanker , vide at sette sig i et livligt Ver clforhold til ham ; han maa bere med sig visse Anskuelser og Interesser og meddele dem til Andre , for saaledes at faa se disse meddelsomme igjen . Om Nogen til Erempel vil soge at gjore sig fortrolig med de rcligiose Forholde i en Bygd , saa maa han selv vere religiossindet og vel forfaren i Guds Ord og saa i Samtale med dem , han moder paa sin Vei , vise Lyst og Gave til at underholde sig om alvorlige Ting , ja til at formane og opbygge , og jo mere oprigtig og nidtjer han selv er , dcsto mere ville Folkets Hjerters Tanker aabenbare sig for ham . — Onstcr Nogen at staffe sig Kundstab om , hvordan Fattigvesenet har udviklet sig paa et Sted , hvordan Forholdet er mellem <b>Fattige</b> og Rige , saa maa han forud have Skjon paa.de vrange Begrebcr og Forviklinger , som i dette Forhold let kunne opstaa , og vere besjelet af Lyst til at se alle Misligheder rettede , og han vil mode Fortrolighed i samme Grad , som han ved sin Tale allerede viser sig at vere fortrolig mcd Alt . — Der er , for endnu at anfore et Erempel , en Mislighed, som i mange , ja i de fleste Egne i vort <b>Land</b> kleber ved den lavere Almues huslige Liv , nemlig en hoi Grad af Urcnlighcd, som navnlig ogsaa af den Grund er saa fordervelig for det <b>moralske</b> Liv , at den maaste mere end noget Andet vedilgeholder en bred Kloft mellem de holere og lavere Klasser ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008121100032'target=_>Kappe, Ernst, 1877, Kortfattet Verdenshistorie for Folket, eller Fortællingeraf Historien</a><h4>1398</h4>denne Fædrelandets unge SM ; fra de theologiste Studerende fik han en SMuklllk og en Disk og fra et Par gamle ZGgtefolk et DMefad . Den 5. Juni forlod Schreuder Norge , og efter et Ophold i London og Kapstaden steg han Nytaarsdag 1844 iland ved Port Natal , dm engelske Koloni blandt Kafferne . Blandt Zulufolket har Schreuder i over 30 Aar virket Alt , hvad Lcrrdom , Mennesiekjcerlighed og Gudsfrygt formaar . Han er fuldkommen Herre over Zuluernes Sprog og har erhvervet sig Respekt saavel hos det raa Folk som hos dets barbariske og grusomme Fyrster . Denne Holagtelse har hidindtil beskyttet ham og hans Medarbejdere under enhver Borgerkrig . Vistnok mener Schreuder , at Zulufolket fom <b>Nation</b> ikke tommer til at omvende sig ; men hvad der dog allerede er udrettet , giver os lysere Forhaabninger om Forøgelse af Tallet paa de enkelte Sjoele , som ville blive frelste . Schreuder giftede fig i 1858 med Emilie Adelheid LjZwenthal fra København . Schreuder har nu uafbrudt arbeidet blandt dette Folk med Undtagelse af , at han 1848 gjorde en Reise til Kina i den Tante at fiytte fin Missionstjeneste did ; men her kunde han paa Grund af sit „ lyse Haar " ikke faa nogen Indgang . Den anden Afbrydelse i hans Virksomhed var , da han i 1866 med sin <b>Hustru</b> aflagde et Veftg i <b>Hjemmet</b> , hvor da paa famme Tid hans Udncevnelse til Biskop fandt Sted . Lcengfelen efter hans kjcere Zulumenighed var dog stor , og den Icengtes ogslla efter ham ; han drog derfor strax tilbage for nu Resten af sit Liv som Overhyrde at virke for Guds Rige . Det var under store Betænkeligheder Schreuder indtraadte i denne ny Virksomhed; han kom ogflla fnart til at erkjende , at de ham nu vaahvilende Pligter vare for tunge , og han nedlagde derfor i 1873 den Hyrdestav, som han kun i 7 Aar havde baaret . Han arbeider altsaa nu ikke lcengere i det norske Missionsselsiabs Tjeneste , hvorimod han er traadt i Forbindelse med den svenske Kirkes Missionsstyrelse for fremdeles blandt Zuluerne „ at kundgøre Kristi Evangelium for de <b>Fattige</b> og helbrede dem , fom have et sønderknust Hjerte " ( Luk . 4,18 ) . Han arbeider nu sammen med den svenske Missioncer Otto Witt , som efter en tre Maaneder lang Reife ankom med <b>Hustru</b> til Biskop Schreuders Missionsstation i Sydafrika i Mai 1876. Bidrag til Schreuders Mission indsamles ved „ Komiteen for den norfte Kirkes Mission ved Schreuder ; " denne Komite har ogsaa paabegyndt Udgivelsen af et „ Missionsblad " .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014022708220'target=_>, 1868, Norsk læsebog</a><h4>5232</h4>kastede , saa at man derfore kunde ansee keele Ny-Kirke- Sogns Folk og det øvrige af Stranden i visse Maader , som en anden <b>Nation</b> , og Staden bestaaende af tvende Partier , hvoraf det eene gjorde Commentarier og Gloser over det andets saa vel Klæde-Dragt og <b>Sæder</b> , som Tale-Maader , Saaledes bleve i min Ungdom Borger-Folk af de andre Spottviis kaldne Peter af det Hollandske Ord Petie eller Faster , efterdi de kaldte deres Faders Søstere og Morbrødre Peter og Ohmer . Borger-Folk forsømmede sig ikke paa deres Side ogsaa at giøre Critiqver over de andre saa kaldne fornemme Folk , hvilke de ansaae som <b>fattige</b> og hoifærdige , og til Vederlag kaldte dem Top-høner , efterdi de bare Sætte og Fontanger , som var en contrabande Klæde-Dragt paa Stranden, hvor Borger-Koner , samt <b>Piger</b> og Døttre bare Huer eller rættere Hætter af en særdeles Facon , hartad giorte som de Straa-Hætter Fruen-Timmeret bruger om Sommeren paa Landet . De samme kaldtes gemeenligen Skur , og bestode af fiint Lærret . Mand-Folkenes Dragt var og heel simpel , saa at de fleeste Borgere saae ud som Skippere , bærende Kioler uden Haand-Klapper .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015012208070'target=_>, 1880, Norge</a><h4>2449</h4>Saasom Bergens Indbyggere ere en Samling af alle Nationer , saa ser man dem meget udi Talemaader , Skik og <b>Sæder</b> at differere fra andre Norske . Og er der ingen <b>Nation</b> , som de mere have udkopieret udi Sprog , Tarvelighed og Arbejdsomhed end den hollandske. Borgerfolk havde for 30 Aar siden deres egen Dragt , hvilken de bestandigen holdt vedlige , uden at rette sig efter fremmede Moder , saa at man paa Klæderne strax kunde kjende en Borger-Kone eller Dotter fra Folk af anden Stand ; thi Magistrats- Personer , Gejstlige og Folk af militaire Stand fulgte de almindelige Lande-Moder og alle de Forandringer , som de ere underkastede , saa at man derfor kunde anse hele Ny-Kirke- Sogns Folk og det øvrige af Stranden i visse Maader som en anden <b>Nation</b> og Staden bestaaende af tvende Partier , hvoraf det ene gjorde Commentarier og Gloser over det andets saa vel Klædedragt og <b>Sæder</b> som Talemaader. Saaledes bleve i min Ungdom Borgerfolk af de andre spotvis kaldne Peter af det Hollandske Ord Petie eller Faster , efterdi de kaldte deres Faders Søstere og Morbrødre Peter og Ohmer . Borgerfolk forsømmede ikke paa deres Side at gjøre Critiquer over de andre saakaldne fornemme Folk , hvilke de ansaa som <b>fattige</b> og hofifærdige , og til Vederlag kaldte dem Top-Høner , efterdi de bare Sætte og Fontanger , som var en contrebande Klædedragt paa Stranden , hvor Borger-Koner samt <b>Piger</b> og Døtre bare Huer eller rettere Hætter af en særdeles Fagon , hartad gjorte som de Straahætter, <b>Fruentimmeret</b> bruger om Sommeren paa Landet . De samme kaldes gemenligen Skur og bestode af fint Lærred . Mandfolkenes Dragt var og hel simpel , saa at de fleste Borgere saa ud som Skippere , bærende Kjoler uden Haandklapper .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013031324028'target=_>Holberg, Ludvig, 1857, Ludvig Holbergs trende Epistler til ***</a><h4>777</h4>I mit Fcederneland findes mange Mcend og <b>Qvinder</b> , som aabenbare tale om deres Omgang med Trolde og underjordiske Folk , og svcerge paa , at de have vceret nedtagne i Banker og Bierge-Hnuler af underjordiske Folk . Disse Menneskers Taabelighed , som haver givet Anledning til Digtet , belees i den fingerede Helt Klim . De Portraits , som hist og her findes i Verket , ere saa mange og saa adskillige, at de udgipre et heelt Comvendium af den <b>moralske</b> Philosophie , saa at hver Side behpver en Npgel . Jeg tilstaaer, at det i visse Maader kand kaldes en Satyre for den skarpe Stiil , hvormed Lasterne afmales . Men , da der skydes i Fleng paa det heele menneskelige KiM , er det en Satyre , som kand anstaae en Philosophus , og maastee andre vilansee Stiilen alt for tvungen , frygtsom og krafteslps , efterdi stige Skrifters Natur udfordrer , at de stal vcere starve , og baade sticere og fornvye . Dog maa Skribentere vogte sig for at gaae over Grcendserne , at de ikke angribe nogen i Særdeleshed , men det heele menneskelige Kipn i Almindelighed . Naar de tage dette i agt , da skriver de ikke Satyrer , men undervise Folk , da saarer de ikke , men de lceger , jo almindeligere en Satyre er , jo mindre fortiener den det Navn , det er meere tilladeligt at igiennemhegle det heele menneskelige Kion , end en <b>Nation</b> allene , meere tilladeligt at criticere en heel <b>Nation</b> , end en <b>Familie</b> ; endelig er den Satyre taaleligere , der beleer en heel <b>Familie</b> , end den , der siger : Rufinns lugter som en Apotheker , og Gorgonius som en Svine - Hyrde . Denne underjordiske Reyse kand med Villighed henfores til Philosophifl Skiemt , som altid haver vceret tilladt , helst da de deri cmfprte Portraits ere saadanne , at de kand passe sig paa hver <b>Nation</b> .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013012424002'target=_>Holberg, Ludvig, 1858, Hundrede og tyve af Holbergs Epistler</a><h4>2208</h4>I mit Fcederneland sindes mange Mcend og <b>Qvinder</b> , som aabenbare tale om deres Omgang med Trolde og underjordiske Folk , og svcerge paa , at de have vceret nedtagne i Banker og Bierge-Huuler af underjordiske Folk . Disse Menneskers Taabelighed , som haver givet Anledning til Digtet , belees i den fingerede Helt Klim . De Portraits , som hist og her findes i Verket , ere saa mange og saa adskillige, at de udgiore et hcelt Compendium af de « <b>moralske</b> Philosophie , saa at hver Side behpver en Npgel . Jeg tilstaaer, at det i visse Maader land kaldes en Satyre for den skarpe Stiil , hvormed Lasterne afmales . Men , da der skydes i Fleng paa det heele menneskelige Kipn , er det en Satyre , som land anstaae en Philosophus , og maastee andre vil ansee Stiilen alt for tvungen , frygtfom og krafteslps , efterdi slige Skrifters Natur udfordrer , at de stal vcere skarpe , og baade sticere og fornpyc . Dog maa Skribentere vogte sig for at gaae over Grcendscrne , at de ikke angribe nogen i Særdeleshed , men det heele menneskelige Kipn i Almindelighed . Naar de tage dette i agt , da skriver de ikke Satyrer , men undervise Folk , da saarer de ikke , men de lceger , jo almindeligere en Satyre er , jo mindre fortiener den det Navn , det er meere tilladeligt at igiennemhegle det heele menneskelige Kipn , end en <b>Nation</b> allene , meere tilladeligt at criticere en heel <b>Nation</b> , end en <b>Familie</b> ; endelig er den Satyre taaleligere , der beleer en heel <b>Familie</b> , end den , der siger : Rufinus lugter som en Apotheker , og Gorgonius som en Svine - Hyrde . Denne underjordiske Reyse kand med Villighed henfprcs til Philosophist Skiemt , som altid haver vceret tilladt , helst da de deri cmfprte Portraits ere saadanne , at de kand passe sig paa hver <b>Nation</b> .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009071303012'target=_>, 1863, Bibelen, eller Den hellige Skrift</a><h4>1879</h4>nu vender tilbage , treder Herren atter for Sangerens Sjel som den , der drager gjennem Dvkenen , Paany holder sit Indtog i Kanaan , ikke i bogstavelig Forstand , da der mellem Ammon og luda ingen Drkcner laa , men a and eli gen som den , der altid forer sit Foll ud af Kampe , Lidelser , Savn og Provelser til Seier og Glede . At bane Veien for dette hans ' lndtog vil sige med ydmyg Taknemmelighet » at anerkjende hans store Naade . — Gud bringer de Enlige i Huus , 7 udforer dc Vuudne til idel Lykke . Herrens Tog gjennem Vrkenen til sin Volig erindrer Sangeren , om hvorledes han overhovedet antager stg de Hjelpelose og Forladte , der ingen anden Tilflugt have at soge end den , der er at finde hos ham , medens de , som falde fra ham , forsmegte . Denne Tanke gjennemfores i Billeder af Faderlose , Enker , Forlade og Fangne i enkelte Ezempler , der fremstille Herren som den milde , kjerlige , omsorgsfulle Hersker paa Zions Bjerg . — Din Skare 11 bosatte sig der . Denne Skare betegnes strax efter som „ den Elendige." Guds Folk bestaar nemlig , nåar det kommer til en rigtig Erkjendelse, af Elendige og <b>Fattige</b> , der Intet eie hos sig selv , men have alt Godt fra Herren . Herren giver Scicrsord ; de , som bringe Glcedesbudssab , ere 12 en stor Skare . Her begynder et nyt Afsnit , idet Betragtningen grunder sig paa Follet , som det der har laget Kanaan i Besiddelse . Det ene Seiersbudstab ( ester det andet ) kommer fra Herren , og altid fremstaar nye Budberere , der forkynde Folket Frelse , d . e . Seier . Her maa man tenke paa den Skik , at Kvintene i Triumf med Sang og Paukcslag forkyudte Seieren og saaledes feirede dens Fest iblandt Folket . De forste Spor til en saadan Seiersfest forekomme 2 Mos . 15 , 20 ; for en saadan var vel ogsaa Deboras Sang ( Dom . 5 ) bestemt ; paa denne festlige Maade kom leftas Datter sin Fader . imodc ( Dom . 11 , 34 ) ; saaledes feirede <b>Kvinderne</b> Davids Seier over Goliath ( 1 Sam . 18 , 6. 7 ) ; at Filisternes Dotre stulde nyde en saadan Triumf efter Sauls Dod var Gjenstand for Davids Frygt ( 2 Sam . 1 , 20 ) . Det ydmygt taknemmelige , troende Tilbageblik paa Guds Riges Historie seer heri en lang Rekke ai Seiere , som Herren har vundet over sine Fiender , horer uastadelig Triumfsange tone igjen , da Herren altid har vendt Klagen til Frydesang . - ^ Hcrrssarcr.nes Konger fly , dc fly , og 12 Husets Bebocrinde udd " elcr Bytte . Den forste Deel af Verset minder om Deboras Sang , Dom . 5 , 19. Den anden Deel tyder paa den etter tilkjempet Seier vundne No , hvori Husfruen fordeler Byttet i <b>Familien</b> . — Naar I hvile nicllcm Kvceghcgncne ( saa ere ' I ) soml . en Ducs Vinger 0. s . v. Luther : „ naar I hvile mellem Grendserne . " . Naar Israel hviler ester den Teier , der har bragt det i Besiddelse af det forjettede <b>Land</b> , seer det ud som en Flok Duer , i hvis skjonne Farver Solen spiller . - Naar den Almcegtige adspreder Konger i_ deri , da snccr det paaZalnion . . . Deri " d . ' e . i Landet , se V . 11. Zalmon betyder  d et Skyggefulle " , et med tyt Skov bevoxet Bjerg i Nerheden af Sikem . Dom . 9 , 48. Selv det morke Bjerg bliver bvidt , nåar Seieren over Kongerne er tilkjempet . Bande Billedet om Duerne og dette om Sneen betegner den Lykke , som Herren , efter at

<h4>3529</h4>Kjarlighed , , hvilken Ingen kan bare til en stor Skare . Saaledes oploser sig ogsaa i den prophetiste Fremstilling Jomfruen eller Wgtehustruen Israel i sine Tusinde , f . Ex . Ez . 16 , 57. 58. Den Elskede fremtråder her som den , der paa Kongens Kaldclse tranger ind i hans Kammer med et stort Antal af Legesostre . Smlgn . Billedet af en Stad som Moder til dens mange Indvaanere . Tilscetningen :  o sirt gli gen " o . s . v. hentyder paa det Rene i denne Kjarlighed , det hellige . Forhold , der Intet har med nogen Sandselighed at gjore . Sort er jeg , dog yndig , I Jerusalems Dotre ! som Kcdars Telte , som Salomos Tclttcppcr . ' Hun drager gjennem Paladsets Sale , hvorigjennem Veien gaar til Kongens Kammer , og gjor Undstyldllinger for sit Udseende hos Kongehusets Jomfruer . Hvorledes kommer denne solbrandte Landspige til dette Sted ? Med Hensyn til sin Sorthed fatter hun sig selv saa lavt som mnligt , idet hun sammenligner stg med en <b>fattig</b> Kedarenerhordes Telte ( 1 Mos . 25 , 13. 1 Kron . 1 , 29. ) , hvilke i sin solbrandte Tilstand gjore et modbydeligt Indtryk ( dette arabists Drkenfolk var i en ualmindelig Grcch forhadt hos Israeliterne , Ps . 120 , 5. ) . Ved Siden Heras fremhcrver hun dog sine Fortrin ved at sammenligne sig med Salomos pragtfulde Forhcrng . Der hentydes her paa den vcesentlige Modscetning , der ligger i Herrens Menighed paa Jorden . Ved forste Viekast trade dens store selvforstyldle Mangler og Ufuldkommenheder Beskueren imode og nedsatte den under mange andre jordiste <b>Samfund</b> , men ved narmere Bekjendtstab aabenbares dens kongelige Herlighed , som den skjuler under dette Idre . — Seer ei paa , at jeg er saa sort de satte mig til Viingaardcns Vogtcrinde , min Viingaard , den som var min , bevogtcdc jeg itle . Her fremfores Grunden til hendes Sorthed . Hendes Moders Sonner ( d . e . ikke hendes rette Brodre men Halvbrodre ) have for at bringe hende under Trceldom og paalagge hende et uudholdeligt Arbeide sat hende til at vogte Viingaardene , men ikke engang den Viingaard som oprindelig tilhorte bende selv , har hun vogtet . Hendes Halvbrodre , Sonner af samme Moder , men ille samme Fader , ere de nagle , blot udvortes Lemmer af hendes <b>Samfund</b> , der ere opfyldte af Fiendstab mod de agte , hvilke de soge at undertrykke . I den jodiste <b>Familie</b> havde — nnder Faderens Levetid — Brodrene en stor Myndighet » over Sostre ( se 1 Mos . 24 , 50. Anm . ) , og nåar disse vare Ha.vsoftre , blev den ofte i hoi Grad misbrugt. Paa saadan Maade bar ogsaa de uagte Menighedslemmer sig ad mod Menigheden i dens Helhet ) ved at indfore den i jordiste Sysler , der bleve den for tunge , saa den ilke engang kunde rogte med Trostab Gjerningen , som Herren havde betroet den . Fortlel mig dog dn , som min Sjcel elsker ! hvor vogter du ( Hjorden ) ? hvor lader du den hvile om Middagen ? thi hvorfor stal jeg vlrrc som en <b>Kvinde</b> , der tilhyller fig , ved dine Mcdbrodres Hjorde ? Man antager enten , at her begynder et ganste nyt Afsnit , hvori den fra den Elskede noget fjernede Brnd , som ikke paa lange har varet inderlige » forenet med ham , staffer sig Knndstab om hans Opholdssted for at knytte Forbindelsen med ham endnu fastere

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112201009'target=_>Prytz, A.F.W.J., 1880, Livsbilleder fra Kristi Kirkes Historie</a><h4>1084</h4>og at der var mange , mange Sjcele , som havde fslt Mangelen , og som med Glcede vilde gjsre Sit til for at raade Bod derpaa . Et stort Antal Prester saavel i hsiere som lavere Stilling udtalte sil : fulde Billigelse af , hvad der var udtalt i Speners ? ia 6 e3i6Sria ( Fromme Vnste ) , og bsde ham broderlig Haanden til fcelles Arbeide mod det store Maal , fremme af en livskraftig Kristendom . At der paa den anden Side udkom alskens Smcedestrifter og onde Rygtcr , var en Selvfslge , aldenstund Spener saa uforbeholdent gik los paa Skinkristendom og dsde Former , og for hans eget Vedkommende havde denne Forfslgelse den gode Virkning , at han gav mere Agt paa sig selv og gik forsigtigere frem , og saalangt fra at bersve ham Modet , gjorde det ham kun trsstigere . Saa blev han nu ogsaa glcrdet ved mangt et Budskab om , at den udftrsede <b>Sced</b> havde begyndt saa smaat at spire .  Ak , at det , som vi se her og der , maatte verre Skud , der bebude det snart kommende Foraar ! " skrev han da . Den nye Bevcegelse spredte sig omkring — fra Hus til Hus , fra <b>Familie</b> til <b>Familie</b> . Den greb Menneskene bande ide lave Hytter og i Slottene . Over det hele Tyskland gik der alskens Rygter om Spener og hans Lige sindede , — at de havde indfsrt fuldstcendigt Acrllesskab i Gods , — at i deres Forsamlinger baade Mamd og <b>Kvinder</b> prcedikede , — at alle Speners Til hcengere istedenfor den sedvanlige Dragt bare Linklceder , — at <b>Kvinderne</b> forssmte Husholdningen og i Sammenkomsterne sammen med sine Veninder fortalde alflens Lcetkerier , — at Spener var afsat fra sitEmbede — og lignende . Men det blev ikke kun ved disse onde Rygter . Speners Svoger , Horbius , blev afsat , fordi han hensynslsst prcrdikede og virkede i Speners Aand . Spener selv truedes med Anklage , fordi han roste mystiske Skrifter , der i sin Udtryksmaade fjcernede fig fra det orthodokse Sprog . Man paastod desuden , at han ivrede for Helliggjsrelsen paa Retfcerdiggjsrelsens Bekostning , og at han bersvede de Anfcegtede al Trsst . Men Spener vedblev rolig at arbeide videre uden at agte paa alt dette . Som et Middel , hvorved der kunde raades Bod Paa den herskende Ugudelighed blandt de Kristne , anbefalede han , at de alvorlige Menighedslemmer skulde slutte sig sammen til mindre <b>Samfund</b> , hvorfra da en Livets Aand vilde blcrse ud til de dsde Menighedslemmer . De Orthodokse havde kun tcenkt paa Bevarelsen af den reneLcere , de havde holdt Vagt paa Guds Stads Mure og Volde mod Angreb udenfra ; men de havde ikke tcrnkt paa den Pest og Hunger, der raadede indenfor Murene . Bekjendelsesskrifterne vare offentlige Troens Vidnesbyrd for Ven og Fiende ; men de vare ikke ufeilbare og ingen Troens Regel , der kunde scettes ved Siden af Skriften . Ligesaa fri stod han ligeoverfor Luther . Han kjendte Luthers Skrifter nsie , han satte dem meget hsit ; imidlertid var der dog ogsaa i dem Feil og Skrsbeligheder som i ethvert menne skeligt Vcrrk . — Spener var overmaade mild ligeoverfor andre Konfessioner . Guds Naaderige strakte fig vidt ud over den lutherske Kirke , omendskjsnt det stod fast , at intet andet Kirkesamfund kunde sammenlignes med den , hvad angik Lcerens nsie Overensstemmelse med Guds Ord . — Ligeoverfor Katholi kerne var han dog meget kraftia . i fine Udtalelser ; thi han fandt hos dem , hvad han scerlig havde meget imod : menneskeligt Magtsprog i Troessager , den vilkaarligste Forvanskning af de kristelige Sandheder og Gjerningshellighed i Livet . Han saa i det pavelige Rom Johs . Aabenbarings  Babel . " Hvad Paulus engang sagde om sia . selv , nemlig at han havde dagligt Overlsb og Bekymring for alle Memgheder , det kunde ogsaa passe paa Spener ; ogsaa om ham kunde det hede :  Hvem er skrsbelig , nden at ogsaa jeg er skrsbelig ? Hvem bliver forarget , uden at det brcrnder mig ? " Dertil kom

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010011903026'target=_>Becker, Karl Friedrich, 1851, Karl Frederik Becker's verdenshistorie</a><h4>1246</h4>Endnu altid bleve Islcenderne drevne til den sydlige Verden as en nesten instinctmessig Hang til at reise . Det var en Mre , naar Nogen havde reist vidt omkring og knnde fortelle om fremmede Sceder . len gammel islandsk Bog hcdder det :  Z3de og usselt er von Liv , naar vi forblive her paa Vjergene , i Dalene og i Arkenerne og aldrig komme til Andre , eller Andre komme til os . " Og i  Kongespeilet " , et andet Skrift , laser man :  Vil Du vere fuldkommen i Videnskaber , saa lcer alle Sftrog , men iscrr Latin og Welsk , thi disse Sftrog gaae videst . " Der bliver ogsaa en Ingling indforl saaledes talende : » Da jeg nu er i mine bedste Aar , vil jeg som Kiobmand sec andre Lande , thi jeg vover ikke at begive mig til Hoffet , forend jeg forft har lcrrt at tiende andre Folkeslags <b>Scrder</b> . " Islcrndercn Bjorn reiste tre Gange til Rom , tog sin <b>Kone</b> Olufa med til det hellige <b>Land</b> og holdt en noiagtig Dagbog over sin Reise , hvilken Dagbog vi endnu have . Mange af disse Islendere og andre siandinaviske Pilegrime toge Krigstjeneste i Eyden , i Serdeleshed i Constantinopel , og udmerkede sig ved Tapperhed og Kierlighed til Berommelse . Men faa forbleve i Udlandet , de fleste vendte , rige paaWrc og Gods , tilbage til deres <b>Hjem</b> , for der at ende deres Dage * ) . Kierlighed til Poesie havde Islenderne medbragt fra deres Stammcland . De dyrkede denne Knnst med megen Iver , og hos de nordiske Konger see vi islandske Skjalde som Hofdigtere . Ved denne Gave og denne Aandsretning faaer den <b>fattige</b> Ve , der ligesom ligger uden for den historiske Verden , en merkverdig Betydning i Poesiens Historie . De gamle nordiske Sagn , der i lang Tid ligesom de greske havde forplantet sig fra Munv til Mund , bleve nu oftstrevne . De to beromte Samlinger, der ofte blive nevnte under Navn af den e l d r e og v n g r e E d d a , sysselsatte sig fornemmelig med Odin og Gudcrne . Men denne Drift til at forplante Sagnene er ogsaa modnet til Historieskrivning , og den islandske Lagmand Snorro Sturlesen ( d0d1241 ) har efterladt et fortreffelig ! Verk over den nordiske Historie . Noget tidligere forfattcde den danske S a i ' o Gramma ticus ien ziirlig Latin en Historie om sit

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2007080112001'target=_>Hugo, Victor, 1862, Elendighedens Ofre</a><h4>1036</h4>det gjorde han for Frugterne . Hr. Mabeuf var kommen saa vidt , at han frembragte Pcerer , ligesaa saftige som Pcererne fra St . Germain; det er af een af hans Combinationer , som det synes , at Mirabellen fra Oktober er avlet , fom nutildags er faa bersmt og som ikte har mindre Parfume , end Sommermirabellen . Han gik til Mesfen fnarere paa Grund af en vis mild Aandsretning end paa Grund af Devotion og dcrncest , fordi han elstede at see Mennesteansigter , men hadede Stsi og fordi han lun i Kirken fandt dem forenede i Stilhed . Da han felte , at han burde vcere Noget i <b>Samfundet</b> , saa havde han valgt den Stilling at vcere Klokker . Forresten havde han aldrig vceret saa heldig at komme til at elske nogen <b>Kvinde</b> saa hsit fom en Tulipanlsg eller nogen Mand saa hsit som en Han havde allerede forlcengst tilbagelagt sine iredsindstyve Nar , da en Dag en Person spurgte ham : Har De aldrig vceret gift ? leg har altid forglemt at tcenke derpaa , fagde han . Naa / det stundom hendte ham , og hvem hender Me Sligt ? at sige : O ! hvis jeg var rig ! saa sieede det ikle ved Synet af en ung Pige, saalcdes som Tilfceldet var med Fader Gillenormand ; det sieede , nåar han betragtede en gammel Bog . Han levede alene med en gammel <b>Huusholderste</b> . " Han var en Smule gigtist og nåar han sov , saa Zreb hans gamle Fingre , stivnede og krogede af Reumatismen, krampagtigt i Sengklcrdernes Folder . Han havde strevet og offentliggjort en Flora fra Omegnen af Cauteretz med kolorerede Plancher , et temmelig anseet Vcerk , hvortil han-besad Kobberpladerne og som han selv solgte . Man kom to eller tre Gange om Dagen og ringede paa hos ham i Gaden Mezieres i denne Anledning. Han tjente derved to tusinde Franks om Aaret ; det var paa det Ncermefte hans hele Formue . Skjsnt <b>fattig</b> , havde det dog lykkedes ham ved Taalmodighed , Fornccgtelse og Tid at skaffe sig en

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015082448274'target=_>Landstad, M.B., 1871, Kirkesalmebog</a><h4>3800</h4>mig i denne Dag fra alt Ondt , og gjore det af Naade til dit store NavnS 3 Ere og min arme Sjcels Salighed . Al Herre Jesu ! lad mig idag have Velsignelfe og Glcede af dit Ord ; lad mig med Lyft gaa op til dit hellige Tempel , og tilbede dig i Aand og Sandhed . Giv mig Naade til at bevare dit Ord i et smult og godt Hjerte og bcere Frugt deraf i Beftcmdighed . Lad mig anvende denne hellige Hvile » dag i hellige Tanker , t hellig Omgjccngelse og i et helligt Liv og Levnet , og lad alt , som tan vanhellige dit Navn , vcere langt borte fra mig og mit HuS . Lad din Aands Arbeidt paa min Sjcel lyttes , saa at din Aand maa herske over min Aand , at din Villie maa vcere min Villie , dit Ord min Lyft , dit Rige mit Hjertes Lcengsel . Lcer mig , o Jesu , idag og hver Dag ret at tcenle paa mine Synder , idag og hver Dag at angre mine Synder , idag og hver Dag at aficegge mine Synder ; og i Troen til dig ved den Helligaands Kraft at fornyes i et helligt Liv , saa at , om jeg lever , jcg da maa leve i Herren ; om jeg dor , jeg da maa ds i Herren , og enten jeg lever eller dor , zeg maa hore Herren til . O Herre , gjor vel imod din Kirte , som du af Naade har vlantet iblandt os . Bevar dit Ord og dine Satramenter , som ere de eneste Midler , du har givet os til Salighed . Stat altid hos Kongen og det kongelige Hus med Lykke , Seier og Salighed . Led al vor Ovnghed baade tirtelig og verdslig i din sande Frygt . Velsign vort <b>Land</b> med Fred og RoUghed , vore Lcgemer med Sundhcd , vore Ågre og Enge med Frugtbarhed . O Herre , giv den <b>Fattige</b> Brod , trost den Bedrovede , husval den Elendige , og forlad den Bodfcerdige hans Synder . O Herre , mistund dig over alle Mennester , og vcer mig arme Synder idag og hver Dag naadig . O Gud , bevar min tjcere ( <b>Hustru</b> ' , Husbond , Forceldre , Born og Tyende ) Slcegt og Nenner fra al Sorg og Stade . Vcer du vor Skjold og Stjcerm mod Djcevelcn og alle onde Men » nesler . Lad ingen Ulykke ramme os paa Legem eller Sjcel , Hus eller <b>Hjem</b> , Gods eller Formue . Gjor ilke med os ester vore Synder og betal os ilke efter vore Misgjerninger , men hav Taalmodighed med os . Afvend Straffen og for » mild Noden , som vi for vore Synders Skyld lunne tomme udi ; og nåar den sidfte Nod er forhaanden , da tag Sjcelen i dine Hcender , thi bu forleste mig , Herre du trofastt Oub . Dit Navn vare lovet , priset og velsignet i Evighed Umen .

<h4>3903</h4>Naade ille ophere at arbeide paa mig til Rettelse og Optugtelse. Men straf mig ikte i din Brede ' , og hjemssg mig ikle i denne Nat i din Bredes Harme . Lad ingen Ulykke ramme mig og Mine enten til Sjcel eller Legeme i det tommende Morte , men bestjcerm mig naadelig med ( Husbonde , <b>Hustru</b> , Forceldre , Bern , Hus og <b>Hjem</b> ) Slcegt og Venner fra al Skade og Fare , og lad os faa Lov at henfty under dine Vingers Skygge , og der hvile tryggelig og vel bevarede . O Herre , lad der i denne Nat vcere en hellig , stille Fred udbredi, over alle Hjerter og over alle Huse i denne din Menighed, og medens vi sove , vaage dit hellige Oie over os ! Bevar dit hellige Ord rent og ret hos os , og dine Sakramentet i din Helligdom uforfalsket indtil Verdens Ende , at vi altid maa besegles med Salighedens Tro og Trost i vore Hjerter ! Bevar Kongen og hans Hus , bestjcerm <b>Land</b> og Rige , Foll og <b>Fcedreland</b> fra alt Ondt . Giv Lykte og Velsignelse, giv Fred og Nolighed , giv Sundhed og Helbred , giv Brsd og Fsde ! O Herre , tlced de Negue , giv de Hungrige Mad og de Torstige Dritte , lad Cnters og.Faderloscs Bon trcenge sig igjennem Styerne og ikte lade af , for den kommer frem og sinder Bonhsrelse for din Stol og Throne ! O Gud , forbarm dia over alle Menncster , og hold et naadigt Oie over mig , dtt <b>fattige</b> Barn , mod al Fare og Ulykte . O Herre , lad mig fove roligen , lad mig vaagne gladelig , lad dine hellige Engle vcere min Beskyttelse , Vagt og Bcerge mod Djcevelen og alt Ondt . Og skal denne Nat vcere min Dedsog Afsleds - Nat — thi vi vide ikte enten Herren vil komme om Aftenen eller om Morgenen , ved Midnat eller ved Hanegal — o Gud , saa lad mig have min Troes Lampe tcendt , at jeg maa indgaa med Jesus , min Sjcels Brudgom , i Brudehuset til Guds Verns Forsamling og de hellige Engles mange Tusinde , der at nyde den overflodige Glcede og Herlighcd, som er hos din hoire Haand evindelig ! Den Glcede give Gud Fader mig og alle Guds Vsrn as Naade formedelst Iesum Kristum vor Herre , som med ham og den Helligaand lever og regjerer , een sand Gud , heilovet i Evighed ! Amen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016031548122'target=_>Landstad, M.B., 1873, Kirkesalmebog efter offentlig Foranstaltning</a><h4>4135</h4>mig i benne Dag fra alt Ondt , og gjore det af Naade til dit store Navns Wre og min arme Sjæls Salighed . Ak Herre Jesu ! lad mig idag have Velsignelse og Glcede af dit Ord ; lad mig med Lyst gaa op til dit hellige Tempel , . og tilbede dig i Aand og Sandhed . Giv mig Naade til at bevare dit Ord i et smukt og godt Hjerte og bcere Frugt deraf i Bestandighed . Lad mig anvende denne hellige Hviledag i hellige Tanker , i hellig Omgængelse og i et helligt Liv og Levnet , og lad alt , som kan vanhellige dit Navn , være langt borte fra mig og mit Hus . Lad din Aands Arbeide paa min Sjæl lykkes , saa at din Aand maa herske over min Aand , at din Villie maa være min Villie , dit Ord min Lyst , dit Rige mit Hjertes Lcengsel . Lcer mig , o Jesu , idag og hver Dag ret at tcenke paa mine Synder , idag og hver Dag at angre mine Synder , idag og hver Dag at aflcegge mine Synder ; og i Troen til dig ved den Helligaands Kraft at fornyes i et helligt Liv , saa at , om jeg lever , jeg da maa leve i Herren ; om jeg dsr , jeg da maa do i Herren , og Mten jeg lever eller dsr , ieg maa hore Herren til . O Herre , gjor vel imod din Kirke , som du af Naade har plantet iblandt os . Bevar dit Ord og dine Sakramenter , som ere de eneste Midler , du har givet os til Salighed . Stat altid hos Kongen og det kongelige Hus med Lykke , Seier og Salighed . Led al vor Ovnghed baade kirkelig og verdslig i din sande Frygt . Velsign vort <b>Land</b> med Fred og Rolighed , vore Legemer med Sundhed , vore Agre og Enge med Frugtbarhed . O Herre , giv den <b>Fattige</b> Brod , trsst den Vedrsvede , husval den Elendige , og forlad den Bodfærdige hans Synder . O Herre , miskund dig over alle Mennesker , og vcer mig arme Synder idag og hver Dag naadig . O Gud , bevar min kjære ( <b>Hustru</b> , Husbond , Forceldre , Vorn og Tyende ) Slægt og Venner fra al Sorg og Slade . Vcer du vor Skjold og Skjærm mod Djævelen og alle onde Mennesker. Lad ingen Ulykke ramme os paa Legem eller Sjæl , Hus eller <b>Hjem</b> , Gods eller Formue . Gjør ikke med os efter vore Synder og betal os ikke efter vore Misgjerninger , men hav Taalmodighed med os . Afvend Straffen og formild Noden , som vi for vore Synders Skyld kunne lomme udi ; og naar den sidste Nod er forhaanden , da tag Sjæleu i dine Hænder , thi du forleste mig , Herre du trofaste Gud . Dit Navn være lovet , priset og velsignet i Evighed Amen .

<h4>4220</h4>endnu meget derpaal Herre , hjælp mig til at blive en sand Kristen , fuld af Tro og gode Gjerninger . Omvend mig du , saa bliver jeg omvendt . Hjælp mig til at aflccgge det gamle Menneske , som fordærves ved bedragelige Lyster , og at issre det ny , som er skabt efter Gud i Sandheds Netfærdighed og hellighed . O hjælp mig arme , skrsbelige Menneske , at jeg lnaa gjennemtrcenges af dit Liv og dit Lys , saa at jeg maa voere og vandre som et Lysets Barn , og lade mit <b>fattige</b> Lys skinne for Menneskene , at de maa se mine gode Gjerninger og prise min Fader , som er i Himlene . Gjor mig taalmodig , tjærlig , velvillig og sagtmodig , saa at vi fordrage hverandre , og tilgive hverandre ; ligesom du , o Gud , har tilgivet os i Kristo , saa gjore og vi . Ja , Gud unde os den Glcede , at vi kunne tilbringe vore Dage i Fredens <b>Samfund</b> med hverandre , og omgaaes i Kjærlighed , at Samlivet ikke skulde forbitres ved Usamdrcegtighed , Klv og Avind . Gud bevare os derfra i Naade . Og nu takler jeg dig , kjære Gud og Fader , for en god og velsignet Dag , for din Hjælp til min Gjerning og for dm Varetægt over mig og Mine . Hold nu din Haand over mig i den kommende Nat . Lad alt Ondt vige bort fra mig . O Gud , bevar min kjære ( Husbond , <b>Hustru</b> , mine Forceldre og mine Born ) Slægt og Venner , at de maa have det godt , hvor de ere ncer eller fjern , og om nogen Nsd kommer paa , være trsstige og frimodige . Gud bevare Hus og <b>Hjem</b> fra Ildebrand , Tyvehaand og alt andet Ondt . Giv mig en rolig Nat og en styrkende Ssvn , bevar mig fra onde , skadelige Drsmme , at jeg end ikke i Ssvne fynder imod dig . Ophold dit Ord iblandt os , bevar din Kirke , og giv al vi til Saligheds Gavn kuuue nyde deus Goder og dervei opholdes og forfremmes i et kristeligt Levnet . Gud giv « Kongen og al Landets Ovrighed Visdoms , Raads og Styr < tes Aand til at virke retsindig i det vigtige Kald , fom du os til Gode har betroet dem . O Herre , giv Ared i vore Dage , bevar os fra den blodige Krig , fra en hastig og uforvarende Dsd og fra al Ulykke . Vcer de Syges Trost , de Ne-dlidendes Hjælper , Enkers Forsvar og de Faderlsses Fader . Vcer ncer hos dem , som stride med Dsden , salv Hjertet med din Trsst og giv dem en salig Forlosning . Herre Jesu , i dit Navn lcegger jeg mig til Hvile , lad mit Hjerte blive hos dig , og lad mig imorgen igjen glad opstaa for at love og prise dit hellige Navn . Og naar det stunder

<h4>4239</h4>Naade ikle ophore at arbeide paa mig til Rettelse og Optugtelse. Men straf mig ikte i din Vrede ' , og hjemssg mig ikke i denne Nat i din Vredes Harme . Lad ingen Ulykke ramme mig og Mine enten til Sjæl eller Legeme i det kommende Msrle , men beskjærm mig naadelig med ( Husbonde , <b>Hustru</b> , Forceldre , Vorn , Hus og <b>Hjem</b> ) Slægt og Venner fra al Skade og Fare , og lad os faa Lov at henfly under dine Vingers Stygge , og der hvile tryggelig og vel bevarede . O Herre , lad der i denne Nat være en hellig , stille Fred udbredt, over alle Hjerter og over alle Huse i denne din Menighed, og medens vi sove , vaage dit hellige Oie over os ! Bevar dit hellige Ord rent og ret hos os , og dine Sakramenter i din Helligdom uforfalsket indtil Verdens Ende , at vi altid maa besegles med Salighedens Tro og Trost i vore Hjerter ! Bevar Kongen og hans Hus , beskjærm <b>Land</b> og Rige , Folk og <b>Fcedreland</b> fra alt Ondt . Giv Lykke og Velsignelse, giv Fred og Rolighed , giv Sundhed og Helbred , giv Brod og Fode ! O Herre , llced de Nsgne , giv de Hungrige Mad og de Tsrsttge Drikke , lad Enkers og > Faderloses Von trcenge sig igjennem Skyerne og ikke lade af , for den kommer frem og finder Bsnhsrelse for din Stol og Throne ! O Gud , forbarm dig . over alle Mennesker , og hold et naadigt Oie over mig , dlt <b>fattige</b> Barn , mod al Fare og Ulykke . O Herre , lad mig sove roligen , lad mig vaagne gladelig , lad dine hellige Engle være min Beskyttelse , Vagt og Vcerge mod Djævelen og alt Ondt . Og skal denne Nat være min Dsdsog Afskeds - Nat — thi vi vide ikke enten Herren vil komme om Aftenen eller om Morgenen , ved Midnat eller ved Hanegal - — o Gud , saa lad mig have min Troes Lampe tcendt , at jeg maa indgaa med Jesus , min Sjæls Brudgom , i Brudehuset til Guds Vsrns Forsamling og de hellige Engles mange Tusinde , der at nyde den overflodige Glcede og Herlighed, som er hos din høire Haand evindelig ! Den Glcede give Gud Fader mig og alle Guds Born af Naade formedelst lesum Kristum vor Herre , som med ham og den Helligaaud lever og regjerer , een sand Gud , hoilovet i Evighed ! Amen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011062212001'target=_>Landstad, M.B., 1880, Kirkesalmebog</a><h4>4171</h4>telig og verdsligi din sande Frygt . Velsign vort <b>Land</b> med Fred og Rolighed , voreLegemei med Sundhed , vore Ågre og Enge med Frugt » barhed . O Herre , giv den <b>Fattige</b> Brsd , trsst den Vedrsvede , husdal den Elendige , og forN den Bodfcrrdige hans Synder . O Herre , miflund dig over alle Mennesker , og vVr m : g arme Synder idag og hver Dag naadig . O Gud , vevai min kjlere ( <b>Hustru</b> , Husbond , Forceldre , Born og Tyende ) Slcrgt og Venner fra al Sorg og Skade . Vcrr du vor Skjold og Skjerm mod Djcrvelen og alle onde Mennefler . Lad ingen Ulykke ramme os paa Legem eller Sjlrl , HuZ eller <b>Hjem</b> , Gods eller Formue . Gjsr itke mei os ester vore Synder og betal os itke ester vore Misgjerningar , men hav Taalmodighed med os . Afvend Straffen og formild Nsden , som vi fm vore Synders Skyld kunne komme udi ; og mai den sidste Nsd er forhaanden , da tag SMen i dine Hcrnder , thi du forlsste mig , Herre du tro » faste Gud . Dit Navn vcere lovet , prifet og vel » signet i Evighed . Amen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2007101812001'target=_>Hillern, W. v., 1872, Ved egen Kraft</a><h4>1007</h4>" Nei , thi da var jeg kun Fader og Spekulant , men ingen Statsborger . Jeg har endnu Me glemt , saa gammel som jeg er , ved Siden af egen Fordel tilligc at have <b>Fedrelandets</b> for Die ! Da jeg reiste til Amerika , gjorde jeg det i dets Tjeneste , for at varetage dets Interesfer , og spMg kun enhver Schweizer , hvorledes dette er lyttedes mig . Jeg forstMete der min Formue og gjorde et glimrende Parti , men Altsammen , mine Rigdomme , min <b>Hustru</b> , mine BM — med Undtagelse af Dig tesverre — og mit hele uforandrede Schweizerhjerte bragte jeg med tilbage til <b>Hjemmet</b> . Men Du — " Hr. HMi strakte Haanten afvergende ud imod SMnen , der vilde afbryte ham , og vedblev i stigende Ophidselse , " men Du Frafatdne , Du vil tåge Dit med Dig herfra , Dine Penge , Dine Kundskaber , Dit Hjerte og vil tilbyde det Alt til en Stat , hvor Du blot er en Draabe i Havet medens Du kunde gjM Dit lille , <b>fattige</b> Fedreland meget godt hermed , vil gjM Dig til Borger der histover og opdrage Dine BM til Amerikanere . Og jeg skulde med aaben Haand ndbetalc Dig Din Arv for at Alt kunde blive op og afgjort imellem os og Du saa snart som mnligt fri og lM ? En HMi stnlde give sin SM Midler til at desertere fra Fedrelandet ? Nei og atter Nei ! Saa sandt der lever en Gud , forn fynligt har lenaadet vor Fedrenejord , saa sandt vi Schweizere siden umindelige Tider har holdt sammen og ofret vor Kraft , vort Blod og

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015080627001'target=_>Landstad, M.B., 1875, Kirkesalmebog</a><h4>3930</h4>og verdslig i din sande Frygt . Velsiqn vort <b>Land</b> med Fred og Rolighed , vore Legemer med Sundhed , vore Agre og Enge med Frugtbarhed. O Herre , giv den <b>Fattige</b> BrFd , trM den Vedrsvede , husval den Elendige , og forlad den Vodfcerdige hans Synder . O Herre , mistuno dig over alle Mennesker , og vcer mig arme Synder idag og hver Dag naadig . O Gud , bevar mm kjcere ( <b>Hustru</b> , Husbond , Forceldre , Vsrn og Tyende ) Slcegt og Venner fra al Sorg ' og Skade . Vcer du vor Skjold og Skjerm mod Djcevelen og alle onde Mennesker . Lad ingen Ulykke ramme os paa LeZem eller Sjcrl , Hus eller <b>Hjem</b> , Gods eller Formue . ikke med os efter vore Synder og betal os ikke efter vore Misgjerninger , men hav Taalmodighed med os . Afvend Straffen og formild som vi for vore Synders Skyld kunne komme udi ; og naar den sidste Wd er forhaandenl , da tag Sjcelen i dme Hcender , thi du forliste mig , Herre du trofaste Gud . Dit Navn vcere lovet , prifet oa velsignet i Evighed . Amen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016031548116'target=_>Landstad, M.B., 1871, Kirkesalmebog</a><h4>3370</h4>Tilflugt Mdtil Ulytteu gaar over . Gud , seud mig Hjcrlft af Himmeleu ! Regjer mig , o Jesu , med diu værdige Hel « ligaaud , at jeg i denne Dag intet andet tamter , taler eller gjor , end det , som kan være til dit Navns Wrc ! > Afverg den listige Djævel , at han ilke i nogen Maade forforer mig ! Bevar min Tunge , at den ikke bander eller bagvasker min Nceste , bevar mine Hceuder fra at rsrc ved det , som er for » buden , bevar mit Hjerte fra Urenhed og Uretfærdighed ! Led mig i dine Veie og lcer mig dine Stier . Dn begynde , fremme og fuldkomme det Gode i mig indtil Jesu Kristi Dag ! Hos mig finder du iutet , udeu et angerfuldt Hjerte , lad det Offer behage dig og finde Naade for dit Ansigt . O Gud , lad ikke af at gjore vel imod mig og Alle fjern og ucer , imod Kirke , Konge , Folk og <b>Fcedreland</b> ! Lad alt Ondt , Krig og Ufred , Huugcr og Dyrtid , Ilds « Nod , Bauds- Vaade og anden Fare være langt fra vore Grcendser ! O Gnd , hjælp naadigen alle Bedrovede , alle Syge og Strobelige , alle <b>Fattige</b> og Elendige , tom med din Aands salige Trost til alle sorgende , lidende og stridende Guds Vorn , gjor det godt for alle Dsende , at de maa se Lys i dit Lys , hvor morkt og tnngt det end kan se ud for dem . O Gud , lad ingen Ulykte ramme enten mig eller min ( Husbonde , <b>Hustru</b> , Forceldre , Bern ) Slagt og Venner , Hus og <b>Hjem</b> , Gods og Formue ! Ak , tjære Fader , velstgu mit Kald og mit Arbeide, min Nceriug og Bjerging , og alt det , jeg stal have Gavu og Godt af . Gud give mig at noies med det , jeg haver , og lcere mig at stjonne sandt , at Gudsfrygt med Noi » somhed er en stor Vinding . Gud bevare mig fra Verdens ' kjærlighed og Gjerrighed ! Gud hjælpe miss til en fast og stadig Tro , en god Samvittighed og et godt og trøstefuldt Haab ! Og paa det Sidste , naar jeg stal gaa al Verdens Gang , saa lad , Herre , mig , din Tjener , fare i Fred , lige- » som dn har sagt , at mine Vine maa se din Salighed , som du i Himmelen har beredt for alle Folk , som tro paa din Son lesum Kristnm , hvilken med dig og den Helligaand lever og regjerer , een sand Gild evindelig ! Amen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111903041'target=_>, 1858, hygieiniske Congres i Kjöbenhavn, Juli 1858</a><h4>607</h4>Befolkningen paa Osterfælled maatte være en heel anden end den , der fandtes i Arbeiderboligerne i Christiania ; dette grundede han paa det betydelige Leie-Tab , man havde lidt og endnu bestandig led ude paa Osterfælled , hvortil kom , at en ikke ganske ringe Deel af Befolkningen der ved Vintertid modtog Understottelse af den offentlige eller <b>private</b> Velgjorenhed , hvorved den altsaa stilledes , i Klasse med de egentlige <b>Fattige</b> , hvilket jo ikke kunde være Tilfældet med Befolkningen i Arbeiderboligerne i Christiania . Foretagendet her var jo ogsaa nyt , og man havde derfor ikke kunnet gaae ind paa nogen Sondring af den Befolkning , der tyede til Bestyrelsen . Kunde imidlertid Foretagendet betydeligt udvides , troede Taleren , at man kunde komme til at bygge bedre i mange Retninger for den bedre stillede Klasse af Befolkningen , og derfor ansaae han det for et Gode , at man vedblev efterhaanden ' at faae nye Bygninger , fordi man efterhaanden vilde faae dem bedre i mange Henseender og tillige faae en bedre Deel af Befolkningen , der kunde grupperes efter Leiens Storrelse , som nu her var overmaade lav . Taleren havde forovrigt overbeviist sig om , at disse Boliger havde ovet en meget ' god Virkning saavel i physisk Henseende — i det han havde seet mange Born komme derud som vantrevne og skrophulose , der vare voxede op til sunde Born — som i <b>moralsk</b> Henseende . For en væsentlig Deel troede han , at dette beroede paa Bygningernes Beliggenhcd udenfor Byen , og deri laae Svaret paa Prof . Hornemanns Sporgsmaal , om man skulde bygge i eller udenfor Byen . Han ansaae det som en nodvendig Betingelse ved et saadant Foretagende for den Deel af Befolkningen , man her nærmest tænkte paa , at bygge udenfor Byen , og Grunden dertil laae i , at man maatte tåge Naturen til Hjælp . Osterfælled var saaledes for Beboerne i Arbeiderboligerne derude deres . Dagligstue , deres Bornekammer , og deri laae Kilden til den Sundhed , som for en stor Deel fulgte med disse Boliger . Han var overbeviist om , at inde i Kjobenhavn vilde disse Boliger , selv om de havde mere Luft end de havde , langtfra i physisk Henseende have bragt saadanne Resultater ; den <b>qvindelige</b> og Barnebefolkningen levede næsten ligesaameget paa Fælleden som inde i Husene , og deri laae Kilden ikke alene til deres Sundhed , man ogsaa til deres <b>moralske</b> Forbedring ; i sidste Henseende maatte han fornemmelig meget insistere paa det Moment , at man havde givet enhver Beboer en Have paa 6 Qvadratalen ; thi det var efter hans fulde Overbeviisning den , som havde forbedret mangfoldige Familieforhold . Naar Manden om Aftenen kom hjem fra sit Arbeide, ligesom ogsaa om Sondagen , gik han og syslede i sin Have , og det var en Fornoielse at see det ene Rækværk fremstaae smukkere end det andet , hvilke de med egne Hænder bragte istand ; i deres smaae Haver pleiede de ikke alene Blomster , men ogsaa Kjokkensager til Vinterbrug . Taleren havde seet mangen Mand , paa Veien til Drikfældighed, blive en ganske anden Mand , efterat han derude havde lært at sætte Priis paa den Hygge , som en god og sund Bolig gav , og paa den Adspredelse , som fulgte med at dyrke disse smaae Haver . Han maatte derfor holde paa , at disse Boliger bleve saa frit liggende som muligt udenfor Byen ; thi man vilde aldrig kunne opnaae lignende Resultater som der , ved at kjobe Boliger i tæt bepakkede Qvarterer af Byen , for at omdanne dem til Arbeiderboliger .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111401049'target=_>Arentzen, Kristian, 1879, Adam Oehlenschlæger</a><h4>2074</h4>nam altßaa ikke som det detvdnin ^ Bknlde Ildgangspunkt for en høiere Udvikling , men som en Afslutning, der lader Tilstand afløse Stræben . Det er ogsaa værd at lægge Mærke til , at han Aladdinsagtigt accentuerer det Eventyrlige og Tilfeldige , men ikke har et Ord tilov6i ' B for Viiiisns og 86 ivvick80inli6d6ii8 Moment . Ligesom Oehlenschlåger nu selv paa mange Maader var blevet en anden , saaledes var det Danmark, hvortil han vendte tilbage , heller ikke det , han forlod . De første Aar al dette Aarhundrede , da vort lille <b>Land</b> havde vidst at bevare sin Neutralitet , kunne betragtes som nogle af vore lykkeligste ; men den lange ulykkelige Krig , som begyndte 1807 , svækkede saavel materielt som <b>moralsk</b> vore Kræfter i høi Grad . Det var en , trods alt sit momentane Liv , ulige mere pinlig end opløftende Lidelseshistorie , Oehlenschlåger kom midt ind i efter Udenlandsreisen , under hvilken han paa et af Historiens Høidepunkter havde bevæget sig i større Forhold , mellem Datidens aandfuldeste Nsend og <b>Kvinder</b> ; og han var mindre end de Fleste anlagt til gjennem en slig Trængselens Skærsild at ndvi ^ leß til en kraftig , neißtsdt Karakter og Baaledeß faae Godt ud af Ondt , Fremmen af Hemmen . Hvor forandret Tiden var blevet ved Belt og Sund , aabendårer sig Blaaende i den nye aandeli ^ e Faktor , som havde begyndt at gjøre sig gjældende , medens Oehlenschlåger færdedes i Udlandet . K F . S . Grundtvig, der kun var faa Aar yngre end Oehlenschlåger , var vel , ligesom denne , født under smilende Naturomgivelser, nemlig i det skjønne 8 ulland ; men hans Vugge stod i en streng religiøs Præstegaard , og han kom desuden alt i Barndommen bort fra Hjemmet

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014061838029'target=_>Rydberg, Viktor, 1877, Romerske dage</a><h4>588</h4>Fjernt laa nu de Dage , da „ romersk Husmoder" var et Hædersnavn . Syndefaldet i <b>Samfundet</b> begynder ikke med Eva men med Adam . Længe værner Kvinclen om god <b>Sæd</b> ; hun gjør det af nedarvet Vane , om ej af Drift til det Ærbare . Men nåar Manden har gjort hende til sin Lige i Last , synker hun hurtig dybere end han , og Fordærvelsen er da ulægelig , thi Slægtens Vugge staar ved en skjændet Arne . Dertil var det kommet i Rom . Det romerske Folks Forfald røbede sig mest afstikkende i den romerske <b>Kvinde</b> . De ældste og ædleste Ætter døde ud , de frie Borgeres Skare smeltede aarlig sammen , fordi Mændene frygtede for Ægteskabets Farer . Giftermaal var det Samme som at give sit Navn hen til at vanæres af en udsvævende <b>Hustru</b> . Man foretrak da den frie Kjærlighed , thi at narres af en Slavinde fandt man mindre krænkende end at bedrages af en Ægtefælle . Augustus greb inci med Lovens Haand for at beskytte <b>Hjem</b> og Stat . Men Loven er nåar Moralen er død . Dog er det et Billede , der er værd at mindes : den gamle Imperator staaende oprejst i sin Loge i Skuespilhuset , medens man allevegne fra Riddernes Bænke raaber til barn under den heftigste Gestikulation : „ Ned med dine Ægteskabslove ! " Da Raabet lyder med forøget Styrke , vinker Augustus Germanikus

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111103039'target=_>Becker, Karl Friedrich, 1842, Karl Friedrich Becker's Verdenshistorie</a><h4>1284</h4>Eudnu altid bleve Isleuderne drevue til dm sydlige Verde » af et nesten instinttmesfigt Hang til at reise . Det var en Wre , nåar Nogen havde reist viot omkring og kunde fortelle om fremmede Seder . I m gammel islandsk Bog hedder dtt : „ Ode og usselt er vort Liv , unar vi forblive her paa Bjergene , i Dalene og i Orrenerne , og vi aldrig komme til Andre , eller Andre komme til os . " Og i Kongespeilet , et andet Skrift , leser man : „ Vil Du vere fuldkommm i Vidmsiciber , sta ler alle Sprog , mm ifer Latin og Welst , thi disse Sprog gcice videst . " Der bliver ogsta en Ungling indfort saaledes talende : „ Da jeg nu er i mine bedste Aar , vil jeg som Kiobmand ste andre Lande , thi jeg vover itte at begive mig til Hoffet , strend jeg forst har lert at tiende andre Folkeslags Seder . " Islenderen Bjorn reiste tre Gange til Nom , tog sin <b>Kone</b> Olufa med til dtt hellige <b>Land</b> , og holdt en noiagtig Dagbog over sin Reise , hvilken Dagbog vi endnu have . Mange af disse Islendere og andre siandinaviste Pilegrime toge Krigstjeneste i Syden , i Serdeleshed i Constantinopcl , og ndmerkede sig ved Tapperhet » og Kierlighed til Berommelst . Mm sta forbleve i Üblandet , de fleste vendte , rige paa Wre og Gods , tilbage til deres <b>Hjem</b> , for der at ende deres Dage * ) . Kierlighed til Poesie havde Islenderne medbragt fra deres Stammeland . De dyrkede denne Knnst med megen Iver , og hos de nordiske Konger ste vi islandske Skjalde som Hostigtere . Ved denne Gave og dmne Aandsretning staer den <b>fattige</b> Oe , der ligesom ligger nden for dm historiske Verden , en merkverdig Betydning i Poesiens Historie . De gamle nordiske Sagn , der i lang Tid ligesom de greske havde forplantet sig fra Mund til Mund , bleve nu opstrcvne . De to beromte Samlinger, der ofte blive nevnede nndcr Navn af dm eldre og yngre Edda , sysselsette sig fornemmelig med Odin og Ondene . Men denne Drift til at forplante Sagnene er ogsta modnet til Historiestrivning , og den islandske Lagmand Snorro Stu rle sen ( dod 1241 ) har efterladt et fortreffeligt Verk over deil nordiske Historie . Noget tidligere forfattede dm danske Saro G ra mm at i c us ien ziirlig Latin en Historie om sit

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008052703001'target=_>Scott, Walter, 1837, Bedstefaders Fortællinger, eller Den nyere Skotlands Historie</a><h4>152</h4>Jacob oergrede sig over sine hungrige Landsmcrnds Graadighed og Paalroengknhed , og forargede sig des , Uden over det <b>fattige</b> og usle Udseende , som mange af dem havde ved Siden af de rige Engelffmcend , hvilket bragde Vanoere over det <b>Land</b> , hvortil han selv , ligesaavelsom de , henhsrte . Han sendte derfor Ordre til Regjeringsraadet i Skotland , at de stulde forbyde saadanne paatrcengende Menneffer at forlade deres Fsdeland , i det han klagede over deres <b>Scrder</b> » g Udseende , som noget der maatte bringe Vancere over alle de Indfsdte i Skotland . En Prollamation blev fslgeli ^ t udstscdt i Edingburg . hvori Regjeringen besv ^ rede sig over , at et si ^ Antal Mcrnd og <b>Qvinder</b> af ringe Stand og Vilkaar , uden Ncrrings , vei eller noget at leve af , begave sig ° fra Skotland til Hoffet , somvarncestm opfpldt med dem , til stor Plage for hans Majestoet og til betydelig Vancere sor den stotste <b>Nation</b> ; thi da disse Beilere til Hoffets Gunst , efter alle deres Omdsmme . som saae dem , vare orkes , lose Skelmer eller usle , <b>fattige</b> Stakler , saa opvakte deres Paatrcengenhed og Mangde den Mening > ar

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111903006'target=_>Becker, Karl Frederik, 1851, Karl Frederik Becker's Verdenshistorie</a><h4>1251</h4>Endnu altid bleve Islenderne drevne til den sydlige Verden af en nesten instinctmessig Hang til nt reise . Det var en Wre , nåar Nogen havde reist vidt omkring og knnde fortelle om fremmede Seden len gammel islandsk Vog hedder det : „ Ote og usselt er vort Liv , unar vi forblive her paa Bjergene , i Dalene og i Orrenerne og aldrig komme til Andre , eller Andre komme til os . " Og i . . Kongespeilet " , et andet Skrift , leser man : „ Vil Du vere fuldkommen i Videnstnber , sta ler alle Sprog , men iser Latin og Welst , thi disse Sprog gaae videst . " Der bliver ogsta en Ungling indfort saaledes talende :  Da jeg nu er i mine bedste Aar , vil jeg som Kiobmand ste andre Lande , thi jeg vover ikke nt begive mig til Hoffet , forend jeg forst hnr lert nt kiende andre Folkeslags Seter . " Islenderen Bjorn reiste tre Gange til Rom , tog sin <b>Kone</b> O lufn med til det hellige <b>Land</b> og holdt en noicigtig Dagbog over sin Reise , hvilken Dagbog vi endnu have . Mange af disse Islendere og andre standinaviste Pilegrime toge Krigstjeneste i Syden , i Serdeleshed i Constantinopel , og udmerkede sig ved Tapperhet » og Kierlighed til Berommelse . Men sta forbleve i Üblandet , de fleste vendte , rige pnn Mn og Gods , tilbage tildens <b>Hjem</b> , for der at ende dens Dage * ) . Kierlighed til Poesie havde Islenderne medbragt fra deres Stammeland . De dyrkede denne Kunst med megen Iver , og hos de nordiske Konger ste vi islandske Sljalde som Hofdigten . Ved denne Gave og denne Aandsntning facer den <b>fattige</b> Oe , der ligesom ligger uden for den historiske Verden , en merkverdig Betydning i Poesiens Historie . De gnmle nordiske Sagn , der i lang Tid ligesom de greske havde forplantet sig fra Mund til Mund , bleve nu opstrevne . De to beromte Samlinger, der ofte blive nevnte under Navn af den e l d r e og y n g r e E d d a , sysselsatte sig fornemmelig med Odin og Guderne . Men denne Drift til nt forplante Sagnene er ogsta modnet til Historiesirivning , og den islandske Lagmnnd Snorro Sturlesen ( dod 1241 ) hnr efterladt et fortreffelig ! Verk over den nordiske Historie . Noget tidligere forfattede den danske S nro Gramma ticus i en ziirlig Latin en Historie om sit

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008112403033'target=_>Hersleb, S.B., 1854, Lærebog i Bibelhistorien</a><h4>294</h4>Imod Slutningen af Dommernes Tid i Israel gik nemlig en Mand fra Bethlehem i luda formedelst Dyrtid med sin <b>Hustru</b> Naomi og tvende Sonner til Moabiternes <b>Land</b> . Her dsde Manden efter kort Tid , og hans to Sonner toge sig Hustruer af Landets Dsttre, Arpa og Ruth . Da nu efter 10 Aars Forlsb begge Naomis Sonner ogsaa dode , lcengtes hun efter sit <b>Fcedreland</b> og vendte derfor tilbage fra et <b>Land</b> , hvor hun havde maattet tabe saameget . Hendes Svigerdottre ledsagede hende og vilde , da Afskedsoieblikket kom , begge folge den elskede Moder til hendes <b>Hjem</b> . Naomi vilde imidlertid ei , at de lcengere siulde dele hendes tunge Skjcebne , hvorfor og Arpa vendte tilbage ; men Ruth erklcerede standhaftig :  Forlang ei , at jeg siulde forlade dig ! hvor du er , der vil ogsaa jeg vcere . Dit Folk sial ogsaa vcere mit , og din Gud er ogsaa min — kun Dsden sial siille mig fra dig ! " Begge vandrede nu tilsammen til Bethlehem . Alle gamle Vekjendte strsmmede sammen , Alt i den lille Stad er i Bevcegelse :  Er det ogsaa virkelig Naomi ? " sverger man med glad Forundring .  O ja ! " svarede den ved fordums Minder nedbsiede Enke )  kalder mig ei lcengere Naomi ( Unde ) , men Mara ( Sorg ) ! ng drog jeg bort , men tomhcendet sender Jehova mig tilbage til <b>Hjemmet</b> . " Hesttiden var just nu forhaanden ; den <b>fattige</b> Ruth gik derfor ud at samle Ar og kom ved en Hcendelse paa den rige Boas ' s Ågre , som var en Ncerbestcegtet as hendes Svigerfader . Denne behandlede Ruth med forekommende Godhed , tillod hende at sanke saameget hun vilde og at spise med hans Arbeidsfolk. Forundret sverger Ruth , hvorledes hun , en Fremmed , havde erhvervet denne Godhet  leg har

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014061108248'target=_>Allen, C.F., 1870, De tre nordiske Rigers Historie under Hans, Christiern den Anden, Frederik den Første, Gustav Vasa, Grevefeiden : 1497-1536</a><h4>652</h4>Adelens almindelige Stilling i <b>Samfundet</b> . Den unge Adelsmands Underviisning og Opdragelse , Øvelse i ridderlige Færdigheder . Adelens Sands for Studier og boglig Dannelse . Reiser til fremmede Lande og Tjeneste ved udenlandske Hoffer , især det franske . Reiser til det hellige <b>Land</b> . Adelens voldsomme <b>Sæder</b> , hyppige Drab , indbyrdes Feider . Adelens forskjellige Vilkaar : Stormands-Ætterne , ringere Adel , Smaa-Adel , der undertiden synker ned i Bondestanden . Byrdsstolthed . Nybagt Adel . Ridderværdigheden . Udpræget Rangvæsen, Skudsmaal givet Adelsmanden af Almuen paa Thinge . Adelens Borge , deres Befæstning og øvrige Indretning . Adelens lette Adgang til høie og indbringende Embeder . Erhvervelsen af Len . Adelsmandens Liv , nåar han sad hjemme paa sin Gaard , hans Forlystelser og Sysler med Landvæsenet . Han var ofte en ivrig Jæger . Adelens ægteskabelige Forbindelser . Ustandsmæssige Ægteskaber fordømte og ledsagede af mislige Følger . Forførelser . Indledende Forhandlinger om Indgaaelse af Ægteskab . Giftermaal anden og tredie Gang hyppige . Den adelige Frue som <b>Huusmoder</b> . Ugifte adelige <b>Pigers</b> Forsørgelse i Klostre , de Sysler , som her dreves . Bortførelser , tvungne Ægteskaber , Ophævelse af slige Forbindelser : Birgitte Giø

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010061506023'target=_>Weenaas, A., 1880, Kristelig Kirkehistorie indtil nyeste Tid</a><h4>1651</h4>over Zuluernes Sprog og har erhvervet sig Respekt saavel hos det raa Folk som hos dets barbariske og grusomme Fyrster . Denne Hoiagtelse har hidindtil beskyttet ham og hans Medarbeidere under enhver Borgerkrig . Vistnok mener Schreuder, at Zulufolket som <b>Nation</b> ikke kommer til at omvende sig ; men hvad der dog allerede er udrettet , giver os lysere Forhaabninger om Forogelse as Tallet ftaa de enkelte Sjcele . som ville blive frelste . Schreuder giftede sig i 1858 med Emilie Adelheid Lswenthal fra Kjobenhavn . Ester hendes Dod cegteviedes han for anden Gang den 10 de November 1879 til den . . troende , fromme og saare duelige <b>Kvinde</b> " Frsken Johanne Margrethe Vedeler , som ester hans fsrste Hustrues Dod havde bestyret hans Hus . Schreuder har nu uafbrudt arbeidet blandt dette Folk med Undtagelse as , at han 1848 gjorde en Reise til Kina i den Tanke at flytte sin Missionstjeneste did . Den anden Afbrydelse i hans Virksomhed var , da han i 1866 med sin <b>Hustru</b> aflagde et Besog i <b>Hjemmet</b> , hvor da paa samme Tid hans Udncevnelse til Biskop fandt Sted . Lcengselen ester hans kjoere Zulumenighed var dog stor . og den lcrngtes ogsaa efter ham ; han drog derfor strax tilbage for nu Resten af sit Liv som Overhyrde at virke for Guds Rige . — Det var under store Betoenkeligheder , Schreuder indtraadte i denne ny Virksomhed ; han kom ogsaa snart til at erkjende , at de ham nu paahvilende Pligter var for tunge , og han nedlagde derfor i 1873 den Hyrdestav. som han kun i 7 Aar havde baarct . Han arbeider altsaa nu ikke wngere i det norske Misfionssclskabs Tjeneste , hvorimod han traadte i Forbindelse med den svenske Kirkes Missionsstyrelse for fremdeles blandt Zuluerne  at kundgjore Kristi Evangelium for de <b>Fattige</b> og helbrede dem . som har et ssnderlnust Hjerte " ( Luk . 4,18 ) . Han arbeidede nu sammen med den svenske Missionoer Otto Witt , som efter en tre Maaneder lang Reise ankom med <b>Hustru</b> til Biskop Schreuders Missionsstation Entumeni i Sydafrika i Mai 1876. Bidrag til Schreuders Mission indsamles ved ..Komiteen for den norske Kirkes Mission ved Schreuder ; " denne Komite har ogsaa paabegyndt Udgivelsen af et . . Missionsblad " . Det var denne Mand . som saaledes tilbod den norske Kirke

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010021503048'target=_>Hersleb, S.B., 1858, Lærebog i Bibelhistorien</a><h4>291</h4>Imod Slutningen af Dommernes Tid i Israel gik nemlig en Mand fra Bethlehem i luda formedelst Dyrtid med sin <b>Hustru</b> Naomi og tvende Sonner til Moabiternes <b>Land</b> . Her dede Manden efter kort Tid , og hans to Senner toge sig Hustruer af Landets Dettre. Arva og Ruth . Da nu efter 10 Aars Forlob begge Naomis Senner ogsaa dede , langtes hun efter sit <b>Fadreland</b> og vendte derfor tilbage fra et <b>Land</b> , hvor hun havde maattet tabe saameget . Hendes Svigerdettre ledsagede hende , og vilde , da Afstedseieblittet kom , begge felge den elskede Moder til hendes <b>Hjem</b> . Nnomi vilde imidlertid ei , at de langere stulde dele hendes tunge Skjabne , hvorfor og Arva vendte tilbage ; men Ruth erklarede standhaftig : „ Forlang ei , at jeg skulde forlade dig ! hvor du er , der vil ogsaa jeg vare . Dit Folk stal ogsnn vare mit , og din Gud er ogsnn min — kun Deden stal stille mig fra dig ! " Vegge vandrede nu tilsammen til Bethlehem . Alle gamle Bekjendte stremmede sammen , Alt i den lille Stnd er i Bevagelse : „ Er det ogsnn virkelig Nnomi ? " sperger man med glad Forundring . „ O , ja ! " svarede den af fordums Minder nedbeiede Enke , „ kcttder mig ei langer Naomi ( Ynde ) , men Mara ( Sorg ) ; rig drog jeg bort , men tomhandet sender Jehova mig tilbage til <b>Hjemmet</b> . " Hesttiden var just nu forhaanden ; den <b>fattige</b> Ruth gik derfor ud at samle Ar og kom ved en Handelse . paa den rige Bons ' s Ågre , som var en Narbestcrgtet af hendes Svigerfader . Denne behandlede Ruth med forekommende Godhed , tillod hende at sanke saa meget hun vilde og at spise med hans Arbeidsfolk.' Forundret sperger Ruth , hvorledes hun , en Fremmed , havde erhvervet denne Godhed . „ leg har

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009072800076'target=_>Hersleb, S.B., 1833, Lærebog i Bibelhistorien</a><h4>206</h4>l . ^ den Tid , Dommerne havde Regjeringen , opsiod engang en Hungersnet » i Landet , under hvilken en Mand fra Staden Bethlehem i lude Stamme udvandrede med sin <b>Hustru</b> Naomi og 2 de Senner til Na , boelandet Moab . I dette <b>Land</b> giftede begge Ssnnerne sig med moabitiste Qviltder , men efter 10 Aars Forlsb vare baade Faderen og Vestvende Senner dede , hvorpaa Naomi besluttede at vende tilbage til sit Fedreneland. Begge hendes Svigerdsttre tilbede sig at felge med hende , men da hun fraraadede dem det , blev vel den ene tilbage , den anden derimod , Ruth , vilde ingenlunde forlade den gamle , <b>fattige</b> , af Sorg dybt nedbsiede Svigermoder . Begge kom da til Bethlehem henimod den Tid , Kornet skulde hostes , og da Mose Lov befalede , at en Agers Eiermand ikke skulde i Kornhosien lade de enkelte Ax opsamle , der vare falone ned fra Kornbaandene , men overlade dem til de <b>Fattige</b> , saa gik Ruth ud for at samle Ax til Odhold for sig og Naomi . Efter Guds Villie traf det Pg saa , at hun kom paa en Ager , der tilherte en riig og anseet Mand ved Navn Boas , som var nerbeslegtet med Naomis afdsde Mand . Han lagde Merke til Ruth , modtog hende venlig , og bod sine Folk , at de skulde med Flid lade endeel Ax falde ned , for at hun kunde finde en rigelig Hest . Siden , da han erfarede , at hendes afdede Mand havde veret ham nerbeslegtet , tog han hende til Hustrne overensstemmende med Most Lov , der befalede, at nåar en Mand dede uden Bern , da skulde hans nermeste Slegtning egte Enken , for at Jordlovde» kunde blive i <b>Familien</b> ; thi den skulde da tilfalde hendes fsrstefedte Sen . Ruth blev saaledes sat i Stand


<h2>“Kvinnelig dannelse” [Woman’s bildung]</h2><h3Kvinde_, Dannelse_, Sjel_</h3><h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010061503024'target=_>Scriver, Christian, 1858, ster Christian Scrivers opbyggelige og trøstefulde Sjele-Skat</a><h4>1060</h4>mig til Wien , hvor jeg med min <b>Kone</b> og mine fire Born offentligen betjendte mig til den christne Tro og ved dell hellige Daab blev indlemmet i den christelige Kirke . " Heraf sees nn tilfalde , at Guds Ord , der iudeholdes i de profetiste Skrifter , er et klart Lys og cil guddommelig Kraft , der formaaer at fordrive det menneskelige Hjertes Morke i Låre og Liv , hvortil ogsaa den hellige Apostel sigler , uaar han siger til sin Timotheus : Den hellige Skrift tan gjore dig viis til Salighed ; den er indblast af Gnd og nyttig til <b>Lcerdom</b> , til Overbeviisning , til Rettelse , til Optngtelse ( Underviisning ) i Netfardighed , at det Gnds Menneske maa vorde fnldkommcnt og dygtiggjort til al god Gjerning ( 2 Tim . 3 , 15 — 17 ) . § 15. Og dette er ndentvivl ogsaa Grnnden til at den Hellig <b>Aand</b> , nåar Han taler om den bellige Skrift , Mr det paa en saadan Maade , som om der ingen anden Bog , intet andet Ord fandtes i Verden ( naglet dog Verden er fntd deraf ) . Se , jeg kommer , siger Messias ; i Bogens Rolle er ' skrevet om " mig ( Ps . 40 , 8 ) , hvilket vel Nogle forklare om Davids Psalmer , Andre om den forste Mose bog , atter Andre om alle fem Mosebogcr , men dog de Fleste om hele det Gamle Testament . Den hellige Skrift er Bogernes Bog , hvoraf alle andre Boger , som have noget Gnddommeligt i sig , ndflydc og mod hvilken alle andre , verdslige Boger ere for Intet at agte , hvor herlige og knnstige dc end monne vare . Saa er da Meningen af de ovennaonte Ord : I Gnds Bog , som Gnd har forfattet, i Hovedbogen , som alene viser Veien til Salighed, er strevet om mig . Paa lignende Maade taler Panlns , nåar han siger : Hvad som helst til-

<h4>3106</h4>og forocmdlet til Saft og Kmft hos dem . Derfor smagte ogfta deres Predikener , Skrifter og Taler af den Kilde , hvomf de udflote , nemlig af deres oplyste , fornyede og med den Hellig <b>Aand</b> opfyldte Hjerter . Man har lenge lagt sig efter den Kunst at kunne skrive Gnldbogstaver med en Pen , og de , som sornaae den , holde , den i hoi Priis . Men jeg kan nok sige , at mange gudfrygtige , acmdrige Mend have kjendt denne Kunst i acmtelig Forstand , efterdi jeg seer , at de have forklaret den gode <b>Lerdom</b> fta herligt og foredraget den fta kmftigt og trosteligt , at man maa foretrekke deres Arbeide for alt forgjeng eligt Guld . Den gudfrygtige Arnd taler etsteds om denne Sag med disse Ord : Saadanne af Gud udsmdte og lerte Mennesker gives der endnu ; men Verden agter dem ikke . Deres Bogstaver ete Ildtunger , som der er Kmft i ; de have Me Ord uden Kraft . Hvor mange Hundret » Boger skrives der Me , som der er Intet i ! Og hvor mange smaae Boger blive Ve skrevne , hvori der er lutter <b>Aand</b> og Liv ! § 16. Ligesom nn denne herlige Gave viser sig hos Lererne , saaledes findes der ogfta noget Serdeles hos de gudfrygtige Tilhorere og eeufoldige Troende , idet de nemlig have god Forstand og Smag paa aandelige og himmelste Ting , ihvorvel den Ene mere , dm. Anden mindre . Naar te lese en Bog , som smager mere af Verdens <b>Aand</b> end af Guds <b>Aand</b> , fta merke de det strax , og efterdi de Me spore Ehristi glodende , levende Kraft deri , fta blive de snart kjede af dem . Naar te hore en Prediken , der ved Guds Naade udstyder af dett rette Forstand paa Skriften , af Troens og Christi Kundstabs indre Lys saavelsom af egen Erfaring , fta ere de for deres Hjerter ligesom et mildt Regn for et tort og torstigt Land . Er derimod en Predikant ligesom en vandlos Sky og et bladlest , nfrugtbart Tre ( Ind . V . 12 ) , der kun giver Ord , meu ingen Kraft , fta kuuue de strax merke det paa sit Hjerte . Dm hellige Aitgustiu siger om sin <b>Moder</b> Monika , at hun knnde gjore god Forskjel mellem en guddommelig og en naturlig Drottt . Hun pleiede at sige , forteller han , at huu ved en egen Smag , som hun ikke vidste at forklare med Ord , let knnde skjelne mellem nåar Du , min Gud ! aabeubarede hende Noget i Dromme , og nåar hendes <b>Sjel</b> kun havde naturlige Drom-

<h4>4614</h4>og den gode Lerdoms Ord , hvilken I have efterfulgt ( 1 Tim . 4 , 6 ) . Den evangeliske Kirke har som en kjerlig <b>Moder</b> strax i Eders Barndom ratt Eder Brystet og lestet og opammet Eder med det saliggjorende Ords uforftlstede Melk . I leve i en Kitle , som ingen Mangel har . paa uoget Godt . Deu har Guds rene og uforfalskede , saliggjorende Ord , som den holder sig ner og ftst til og som den retter sig efter i Tro , Levnet , Lidelse og Dod . Dett har ' ikke bygget paa Flyveftnd , ikke paa Meunestesetninger og egne Indbildninger , men paa den provede Grund- og Hjornestcen Christus Jesus , som er hele Skriftens Midtpunkt og Kjerne . Af Gnds Ord lerer den nu selv og lerer Andre , at Meuuestet maa kjende sig selv og den Elendighed , hvori Synden har styrtet det , ftygte for Gnds Vrede , Mge sin Tilflugt til Hans Naade og Bannhjerlighet » i Christo Jesu , ved Troen tomme i Samfund med Ham , finde Syndernes Forladelse , Retferdighed og Salighed hos Ham og ved Haus Blod , <b>Aand</b> , Ord og Naale blive gjmftdt , oplyst , omvendt og ftrnyet . Den formaner ogfta sine Egne med al Trostab og gudelig Nidkjerhed til den styldige Taknemmelighed , til at elste Gud og Nesten , til et helligt , gudeligt Levuet , til Taalmodighed , Wmyghed , "Sagtmodighed, Venlighed , Kyskhet » , Afholdmhed , Gavmildhet » , Noiftmhed , Oprigtighed og Sanddruhed . Den lerer, at man stal bede andegtig ! og inderligt , paakalde den himmelste Fader tned barnlig Tillidsfuldhed og vente og erholde unådigst Bonhorelse as Ham . Den driver paa at vi stulle pryde den rene <b>Lerdom</b> med en reen og ustraffelig Vandel og daglig tåge Christi Kors op og folge Ham . Den giver trostelig Underviisning om Guds Boms Kors , Trengsler og Anfegtelser og veed at aabenbare den Guds Kjerligheds Hemmelighed , som ligger stjnlt derunder . Den , besidder varig , sikler og riig Trost til at ftrsode og overvinde alle dette bedrovelige Livs Moisommcligheder og Gjenvordigheter med. Den lerer , at man stal foragte Verden og dens Forfengelighed , tmgte efter de himmelste og evige Goder og elste og stunte efter den Herres Jesu Christi Aabenbarelse . Dm leter os at gaae Doden villigen og gladeligm unote , og hvorledes vi efter alle hellige Guds Botns Exempel i Tillid til

<h4>4857</h4>bet , og Herren skal borttcige deres Prydelser osv. ( Efti . 3 , 16 flg . ) . Herren vil hjemsoge Alle , som klede sig med fremmed Kledning ( som ikte ere tilfredse med sin sedvanlige Landsdmgt , mm af Hofferdighet ) og Nyhedssygc altid hige ester nye <b>Moder</b> og synes godt om sig selv i fremmede Dragter — Zef . 1 , . 8 ) . Skikker Eder ikke lige med denne Verden ; men bliver forvandlede ved Eders Sinds Fornyelse! ( Rom . 12 , 2 — ville I endelig have Forandring og Nyt , fta seer til at fornye Eders Hjerter og forandre Eders Sind ; thi hvor ofte Verden end stiller <b>Moder</b> og Klededmgt , fta bliver den dog ligesaa fuldt i den ene Skikkelse som i den anden kjotelig , jordisk og ugudelig , og det var godt sagt af eu <b>vis</b> klog Maud , at uaar han ftae et riigt og overdaadigt Menneske fta ofte forandre sin Dmgt , mindede dette ham om at Slang erne vel ofte stifte Ham , men blive dog ligefuldt Slanger). § 40. Iforer Eder den Herre lesum Christum, og pleier ikke Kjodet til at vekke Begjerlighed ! ( Rom . 13 , 14 — Legemet maa vel have fornodent Underhold og tilborlig Pleie med Mad , Drikke , Sovn , Kleder og Deslige ; mm Kjodets Lyster maae dog ikke bestemme <b>Maalet</b> i fta > danne Ting ; ei heller maa man behandle det syndige Legeme saa godt , at det liig en kaad Hest bliver ustyrligt og gjenstridigt mod Aanden ; men Alt maa stee overensstemmende med Guds Ord og Villie , i hellig Frygt for Ham , til den fante Gudftygtigheds Befordring og ikke til dens Forhindring . ) <b>Kvinderne</b> skulle pryde sig i sommelig Kledning med Bluferdighet ) og Tugtighed, ikke med Fletuinger eller Guld eller Perler eller kostbart Kledebon , men , som det sommer sig <b>Kvinder</b> , der beljende sig til Gudsfrygt , med gode Gjeruinger ( 1 Tim . 2 , 9. 10 ) . <b>Kvinderne</b> ( de Troendes ) skulle vandre saaledes , at de , somikke troe Ordet , kunne vindes uden Ord ved <b>Kvindernes</b> Omgjengelse ( den hellige Paulus byder ellers <b>Kvinderne</b> at tie iForsamlingerne ; derimod vil han saavelsom dm hellige Petrus , at de med sit tugtige , ydmyge , erbare og gudelige Levnet stulle vidne om Troens og Christendommms Kraft , og ligesom Mentene ved Ordet , ftaledes

<h4>5422</h4>staaer dm elle Ring af den anden , hvilket derimod ikte steer i det rindmte , fordi det flygtmde Vand , ftln Seneka siger , forstyrrer enhver Figur . Saaledes vgfta : uaar Satan kommer til en orkeslos og vellystig <b>Sjel</b> og bringer en ond Tanke og syndig Begjering ind i dens Sind , fta finde disse ikte blot Rum , mm formere sig ogsaa snart , fta at den ene stedse bliver syndigere end den anden , indtil det udbryder i Gjerning , medens derimod dm , som bestandig er sysselftt med helligt og godt Arbeide , gjor Alle hans Anslag til Intet , fta at han maa trekke sig tilbage med Stam . Jeg har hort forstandige og flittige <b>Huusmodre</b> sige til sine Dotre og <b>Piger</b> : Et <b>Fruentimmer</b> maa aldrig vere ledigt; har hun intet Andet at gjore " , fta maa hun heller klippe en Lap af en Kledning og fye den paa igjen ; dette er en <b>Lerdom</b> for de " gudelstmde Sjele ; de maae stedse bestjeftige sig med gode Gjerninger, foråt de ikke stulle ftae Tid til at tenke paa de onde . Jeg har kjendt gudftygtige Hjerter , som have afstrevet hele gudelige og opbyggelige Boger, forn de ellers nok kunde bast Raad til at tjobe , deels foråt gjore sig tesbedte bekjendte med dem , deels ftr ikte at vere orkeslose . En gndftygtig adelig Frue var meget flittig til at svinte vg sve og sogte at fylde sit Tal ligesaa godt som sine Piger, og da en af disse sagde , at baade hnn seloog mange Andre undrede sig over at Fruen var fta flittig til saadant simpelt Arbeide , vg at mau desaarsag beskyldte hevde for Gjerrighet » , svarede huu : Behover da ikke jeg at arbeide med mine Hender ligeftavel som Andre , deels fornt kunne ede mit Brod nden Synd , deels somt have Noget at give de Trengende , og deels ftr ikke at falde fm Lediggang i Synd ? § 36. Et kysk Hjerte maa 5 ) elske Afholdenhed i Mad og Drikke , undertiden spege det syndige Kjod med Faste og bestandig skye Fylderi og Drukkcnskab , som den nermeste Vei til Utugt . Det er intet Under , at Utugten i vor Tid allevegne har tåget Overhaand , siden Drukkmstabm , denue <b>Moder</b> til al Synd og Ugudelighet » , nesten ikte mere holdes for nogen Synd . Intet er skadeligere og farligere for Kyskheten end et nmaadeligt og uordentlig ! Levnet. Ligesom den Ager , der ftaer formeget Regn , legger sig og opfyldes med Blaatlotter og andet

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009091403021'target=_>Luther, Martin, 1864, Dr. Martin Luthers Udførlige Udlæggelse af første Mosebog</a><h4>123</h4>han stulde blive Gud lig , netop ved et lignende Fald , hvorved Djcevelen selv ogsaa faldt fra Himlen , saa berovede dog Gud ikke Naturen Fodfelen eller Borneavlingen , men tillod , at baade de Gudfrygtige og de Ugudelige avlede Born . Det alene har han villet <b>vise</b> , at det ikke er nok at fodes til denne Verden af Kjodet , men at foruden denne Fodsel , som endnu er forbleven i Kjodet , endnu er fornoden en Gjenfodelse og Gjenfodelsens Fornyelse ved den Hellig-Aand . Til at bevise dette har den hellige Paulus brugt denne Text om Esau og Jakob og ogsaa den om Ismael og Isak . Thi Begge ere de fodte efter Kjodet og efter den forste Fodsel . Men Ingen af dem var fra det menneskelige Nige kommen ind i Himmerige , hvis ikke Isak var bleven sat til Arving ved den nye Fodsel og ligeledes ogsaa Jakob . 74. Gud har brugt disse fremragende og herlige Mcend til at fremstille dem som klare Exempler for det hoffcerdige Folk , forn Abrahams Efterkommere skulde blive . Thi han vidste vel , hvor hoffcerdige de vare , at Ingen kunde tcemme dem , og at de havde Nakker haardere end Diamant , nemlig paa Grund af denne at de vare Abrahams Born og Efterkommere . 75. Derfor vilde han i Texten fcette denne Udvcelgelfe for derved at stoppe lodernes hoffcerdige Mund og tilintetgjore deres Bevis om den kjodelige Fodsel . Thi denne er ingenlunde nok ; men foruden denne kjodelige Fodsel trceves ogsaa Gjenfodelsen . Hvor hellige end disse Fcedre ere , Abraham , Isak og Andre , saa avle de dog ingen Rigets Born ved den forste Fodfel , som er stabt fra Forst af fra Verdens Vegyndelse og fordcervet ved Djcevelen ; men foruden Stavelsen maa ogsaa Kaldelsen komme dertil . Thi saa siger Paulus Rom . 9 , 11. 12. : da de endnu itte vare fodte og ikte havde gjort hverten Godt eller Ondt o . s . v. , itte ved Gjerninger , men ved ham , som laldte , ikke ved ham , som stabte ; thi de bleve begge stabte og undsangede paa eengang . Men sorat hele Verden og fornemlig det haardnakkede Folk kunde vide , at foruden Stabelfen ogfaa Kaldelfen er nodvendig , da blev det sagt til hende , ikke ved Gjerninger , men ved ham , som kaldte : den ALldre stal tjene den Angre . 76. Dette Bevis er fra Verdens Begyndelfe af uaflcidelig bleven indstjoerpet og behandlet , ligesom Tvisten mellem Vrodrene , Esau og Jakob , begyndte i Moderlivet . Og endnu er der ingen Ende paa Krigen . Thi det er ganste den samme Strid , som var mellem Kam og Abel og mellem Slangens Efterkommere og <b>Kvindens</b> Scrd . Den forste Fodsel er hoffcerdig og hovmoder sig saare , fornemlig da , nåar det ogsaa kommer til , at Forceldrens havde en synderlig Anseelse , det er , nåar de vare fodte af Fcedrenes Blod . Siden hjcelper ogsaa Rigdommen dertil , Magten , det verdslige Rige , <b>Visdommen</b> , Retfcerdigheden , Religionen eller Gudstjenesten . Deraf kommer Fiendstab mellem Kam og Abel , Ismael og Ifak , Esau og Jakob , mellem vor Herre Guds og Djcevelens Kirke . Og Djcevelens Kirke har stedse Lyst til at herste og regjere , forst paa Grund af Herkomsten og Fcedrene , som af en retmcessig Aarsag , derncest paa Grund af den timelige Velsignelse . Vi ere , sige de , Fcedrenes Born ; vi ere blevne store og have udbredt vor Magt ved det timelige Regimente , dertil ogsaa ved Religionen , Penge og Gods , ved mange herlige Seire og Undergjerninger ; kort sagt , vi ere saaledes ovhoiede , at alle Folk ere ligesom Halen , vi derimod som Hovedet . Dette Bevis synes at vcere uovervindeligt og uigjendriveligt . Men Skriften siger det

<h4>487</h4>Isak tåget hende om Halsen og kysset hende , bedet hende vcere tilfreds og trostig , og har vel fagt til hende : min kjcere Rebekka , beer dog denne Ulykke med Taalmodighed ; Gud har aabenbaret sig for mig og forjcettet mig , at han vil redde mig og dertil bestytte og bestjcerme mig . Saaledes kunne vi forfvare Ifak og undstylde hvad han har gjort , faaat vi ikke med de urene Svin loderne gjore ham til en Cyniker eller Diogenes . 95. Men jeg vil lade disse Tanker staa derhen , vil ikke drive haardt paa dem og vil heller ikke ganste forkaste dem . Men det fynes dog at vcere fcmdsynligt , efterfom det jo ellers ikke er lysteligt , at En kysser sin <b>Hustru</b> i fremmed Land , hvor han er iblandt Fiender . Men han trcengte til Trost og et Vidnesbyrd om , at Gud var ncervcerende hos hende og havde forjcettet dem at redde og bestytte dem , og dette kunde ste ved denne Kyssen og Legen . 96. Dette er en hcederlig Spog , som vel anstaaer en from , cerbar Wgtemcmd med sin <b>Hustru</b> . Vilde han spoge saaledes med sin Soster eller med sine Husfolk , det vilde ikke anstaa ham vel . Thi der sommer det sig , at man befaler dem , hvad de stulle gjore og lade , og at man er alvorlig , felv nåar man troster dem . Men med hende , som Gud har forenet med mig , kan jeg spoge , lege og tale venlig , sorat leve med hende med Fornuft og <b>Visdom</b> . Som St . Petrus 1 Brev 3 , 7. formaner Mcendene , at de stulle bevise sine Hustruer , som deu svngere Del , Mre , da de ogsaa ere Medarvinger til Livets Naadegave og Moore og den anden Del af Mennesteflcegten , hvorvel den svageste Del , hvad Legemet angaaer . - 97. Vi Mcend ere stcerkere , derfor er der ogsaa anbefalet os storre Gjerninger i vort Embede , i hvilke der dog ofte mcerkes et <b>kvindeligt</b> Hjerte hos os . Men der er dog Forstjel mellem et mandligt og et <b>kvindeligt</b> Gemyt . <b>Kvinderne</b> ere befvcerede med Husarbeide , dertil med Fodfelen og Nornenes Opdragelse , have dertil ogfaa Omforgen for Huset liggende paa sig ; vi derimod have paa os det borgerlige og kirkelige Liv , fordi vi ere stcerkere . Men hvorledes vi forholde os deri , fe vi nok . Ofte fpores der ikke mindre Svaghed i den Bestyrelse , som Mcendene have paa sig . 98. Derfor vil jeg folge denne Mening , at Isak venlig kyssede hende , ikke alene for at troste hende , men ogsaa fornt <b>vise</b> sig imod hende som en from 2 Sgtemnnd , faaledes forn en Saadan pleier at vife Spog og Venlighed mod sin cerbare <b>Hustru</b> , som man ofte kan fe paa de Wgtemcend , som ere noget venlige . 99. Kongen var vistnok ikke saa <b>vis</b> , at han kunde forståa disfe vigtige Aarsager , hvorfor Isak kyssede sin <b>Hustru</b> ; men han forstod , at han dertil havde almindelige Grund , som man vel kan <b>vise</b> offentlig . 100. Og det er faare smukt , uaar en Wgtemcmd ogsaa i ydre Manerer viser sig venlig mod sin <b>Hustru</b> . Derimod er det stammeligt og forargeligt , nåar der er Mcend , der <b>vise</b> sig grcetne , lunefulde , haarde og uveulige imod sine Hustruer og hverken med Ord eller med Gjerninger give noget Tegn paa at de elste dem og ere venlige mod dem . Saaledes hore vi ofte <b>Kvinderne</b> rofe det og glcede sig derover , nåar de se , nt Manden lever venlig og i god Enighed med sin <b>Hustru</b> . O hvilket godt Wgtestab er ikke det ; der er sand Kjcerlighed mellem dem og en salig Endrcegtighed ; det pleie ogsaa Mcendene at rose . Men hvoraf

<h4>794</h4>fove og stole paa dem . Thi paa denne Maade tvivler heller itte Isak paa sin <b>Hustru</b> Rebekkas og sin Son Jakobs Trostab ; han tcenker , at de ere saa fromme og trofaste , at de itte kunne handle listig eller bedragersk mod ham . Skjont han derfor paamindes ved Jakobs Rost , kunde han dog ikke ncere nogen Mistanke mod ham , men mener , at Esau for Spog har forandret sin Stemme ; itte sorat bedrage sin Fader . 68. Og dette er saaledes hele Livets almindelige Gang , ikke alene med de Hellige , men ogsaa i det daglige Liv med alle Mennester , som Exemplerne i Komedierne <b>vise</b> . Hvo der er lettroende , han bedrages ; men hvo der itte er lettroende , ham vil du forgjceves prove paa at bedrage. Men derfor ster der meget Saadant ide Helliges Liv , foråt vi stulle se , hvorledes Gud styrer sine Hellige og af deres Daarlighed og Forvovenhed kan gjore den allerhoieste <b>Visdom</b> ; dertil giver han Lytte til deres Raad og Anslag , som i Begyndelsen se meget taabelige ud , saaat de maa faa en stjon Udgang . Det hcender ofte , at hellige Mennester handle uforsigtig nok , ligefom ogfaa Rebekka her feiler stammelig ; men hun kommer dog igjennem dermed , fordi hun er troende .

<h4>822</h4>alt dette . Jakob er gaaet ind til sin Fader i sin Broder Esaus Skikkelse og har lodne Hcender , som vare tillavede saaledes med List og et fint Bedrag , men dog kan han itte ganste bedrage sin Fader ; thi han kjender dog Jakobs Stemme . Her ser dn nn , at Gud smukt retter paa denne Forvovenhed ved <b>Kvindens</b> Tro , paa Grund af hvilken det Naad og Anstag , forn i sig selv var daarligt , maa blive saare godt og heldigt . 93. Saaledes beskriver nu den hellige Skrift de Helliges Historie , for i den at rofe og prife Troens store Kraft . Derfor , hvo der har Guds Ord , han stal cmfe sig for falig og stal vende Oinene bort fra disfe ncervcerende , timelige Ting og rette dem paa de tilkommende , usynlige Goder . Thi Guds Ord , og fornemlig Forjcettelsen , taler ikke om de ncervcerende , men om de tilkommende Ting , som Ingen har erfaret. Troen hcenger fig ved det , forn endnu slet ikke er til , og venter , indtil det bliver til Alt . Den er en Kuust og <b>Visdom</b> , forn har at bestille med det , som er morkt og slet itte er til ; det er , med saadanne Ting , som den itte har erfaret , forn man itte fer , og forn ncesten ere umulige . Og hvo der vil vcere en Christen , han maa vel betcenke det i sit Hjerte og holde fast derved . Thi alle de andre Kunster lceres ved Slutninger fra provede Grunde og af Erfaringen ; de have itte sin Gruud og Begyndelse i det , forn Intet er , og iscer som man ikke fer , forn er umuligt , urimeligt og taabeligt . Men Troen , forn griber Forjcettelsen og holder sig til den , hcenger med Hjertet fast ved det , forn er ganste urimeligt og umuligt ; den lader sig noie med Ordet og den guddommelige Forjcettelse . 94. Thi saaledes gjor ogsaa Jakob . Moderen befaler ham at modtage Velsignelsen af sin Fader ; da <b>moder</b> det ham , som slet Intet er ; thi Velsignelsen tilhorte jo Esau og itte Jakob ; der modte ham , siger jeg . Ting , som vare ganste umulige . Thi han maatte jo tcenke : hvorledes stulde jeg kunne velsignes ? Velsignelsen tilhorer jo min Broder . Desuden er min Broder lodden , men jeg er glat ; dette er den Fare derved , at jeg kunde bringe Forbandelse over mig og itte Velsignelse. Derfor er umuligt , hvad Sagen i sig selv angaaer ; thi Velsignelsen er ikke min. Min Fader vil velsigne Esau og bekymrer sig i denne Sag slet Intet om mig . Derimod er det vel muligt , at Forbcmdelfen kunde komme over mig og sorblive hos mig , da jeg er flet og glat paa Huden , hvis nemlig Fader vilde mcerke Bedraget . Derfor , min kjcere <b>Moder</b> , vil det visfelig gaa galt med os . 95. Men imod alt dette siger Moderen : kommer der en Forbandelse, da stal deu stade mig og forblive hos mig . Det er en <b>Kvinde</b> , men ingen almindelig <b>Kvindc</b> . Hun er et Menneste med <b>kvindelig</b> <b>Aand</b> ; men faadan Aandskraft er der hos hende , at hun endog overvinder de allerstcerkeste Mcend . Thi hun mener , at hun kuude komme igjennem med sit Forehavende , saaat Velsignelsen blev givet den , hvem den dog ikte tilkom eiter Loven og den almindelige Regel . Hun troer Ting , som ere ganste umulige , urimelige , taabelige og farlige . Hun tvivler itte paa , at Jakob maa blive den Velsignede , hvorvel Alt satte sig derimod og stred lige imod det . 96. Saaledes er Troen og dens Kraft en Hjertets Tanke eller , foråt bruge dette Udtryk , en saadan forudfattet Mening og Forvisning , hvormed huu hcenger sig og holder sig fast ved det , som er Intet , umuligt og ganste urimeligt . Hun har kun det guddommelige Ord ,

<h4>1163</h4>132. Saaledes er dette saare mcerkelige Exempler ; thi de <b>vise</b> ikke alene , at Gud talte med Fcedrene , men bevidne ogsaa , at Gud ikke lyver i sit Ord , om det end ser ud , som om Alt skulde saa et ganste modsat Udfald , og om end Verden er rasende , Kjodet , Modstcmderne og Djcevelen med alle Helvedes Porte rase og larme . Thi han byder , saa ster det . Men det er saare vansteligt for os Troende at bryde igjennem ; thi Djcevelen retter sin grumme Vrede kun ene og alene paa dette , at han maatte saa revet os bort fra Ordet , paadet vi stulle blive trcette og forsagte , enten over den megen Trcengsel , eller fordi det varer faa lcenge , og vi faaledes stulle forlade og forkaste Ordet . Derfor fcettes nu disse to Stykker hos hinanden , nemlig Guds Almagt og det mennestelige Hjertes Svaghed eller Vcmmagt , som stal give Taal , lide og vente saa lcenge , indtil Ordet opfyldes . Saaledes taler Bernhard om Jomfru Marias Tro , da Engelen havde forkyndt hende , at hun stulde blive Christi <b>Moder</b> , at Jomfruens Tro , idet hun kunde tro Engelens Ord , ikke var et mindre Under end Ordets eller Guds Sons Mennestevorden selv . Derfor er det Storste i de Helliges Historie de Ord , som Gud taler med dem . Vistnok stal man prise deres Dyder og Gjerninger , og Gud vil ogsaa have dem ; men man stal stille dem paa nederste Plads ligesom Fodderne . Men Hovedet i de Helliges Liv er Guds eget Ord . 133. Men at Alt faaer et faa urimeligt Udfald og , forn det fynes , det Modsatte ster af , hvad Gud forjcetter i Ordet , det ster paa Grund af Djcevelens Magt og vort Kjods Svaghed , som endnu altid tvivler paa det forjcettede Ord , om det virkelig vgsaa er evigt og sandt . Saaledes stulle vi siden faa hore , at disse store og rige Forjcettelser , at Gud siger til Jakob : jeg vil vcere med dig , o . s . v. , cmfcegtes stcerkt . 134. Vi , der ikke ere i saadan Fare og Vesvcering som den , hvormed Jakob kjcempede , ' lcese dette kun overfladisk og sovnig . Vi mene , at det er en simpel og let Sag ; men vare vi i hans Sted , saa vilde vi vcere ganste anderledes tilsinds . Thi vi , som tro Gud i hans Forjcettelser og ere frelste ved Christi Blod , se , at Alt hos de Christne gaacr saaledes til , som om det var umuligt , saa at Mennesket ganske maa dodes med Kjodet og med al <b>Visdom</b> og Fornuft og alene maa hcenge ved Ordet , forn vel kan fees ogfaa hos de hellige Patriarker . Og hvo der ikke foretrcekker Ordet og scetter det hoiere end sit eget

<h4>1297</h4>32. Derfor , siger jeg , ere ikke alene de store og ridderlige Dyder kostelige og dyrebare for Herren , dertil ogsaa de store , herlige Gjerninger, som han virker ved os , eller de Helliges Blod , Dod og store , svare Kampe ; men de allerringeste Klove ere ogsaa dyrebare for ham . Ja , hor Christum ; han gjor det endnu bedre , Matth . 10 , 30. : men og alle Eders Hovedhaar ere talte ; I stulle ikke miste eet Håar . Kjere , hvad er dog ringere og foragteligere paa Menneskets hele Legeme end et Håar eller en Negl ? Og dog ere de ogsaa talte , og Faderen i Himlen sorger for dem . 33. Paa denne Maade stal man handle om Exemplerne paa ringe og simple Gjerninger hos de Hellige , foråt vi deraf kunne hente <b>Lerdom</b> og Trost . Thi vi negte ikke , at de ere ringe og foragtelige , nåar vi kun betragte det Nedstab alene , ved hvilket de ste ; men man maa fe hen til Gud , forn felv er den rette Udover af dem . Thi hvad enten du ser Urenheden eller ved de Hellige og deres Gjerninger , saa behage de dog Gud vel , saasom han ved dem udretter baade de simple og de fkjonne , herlige Gjerninger ; thi det er Guds Gjerninger , og Gud virker med dem . Derfor er dette en stor , uendelig Trost for dem , forn tro . Og alt dette bestrives , foråt vi stulle fe , hvor hjertelig Gud elster os , hvor gunstig han er os , og hvilken engstelig Omsorg han drager for os . Saa noie fer han paa mig , at han pasfer paa , om jeg mister et Håar . Teller han nu Haarene og sorger for dem , faa sorger han endnu meget mere for mit Legeme , <b>Sjel</b> , Blod og al Lidelfe , forn kan mode mig . 34. Men disse Ting ere altfor hoie , derfor tro vi det ikke , og jo ringere og urenere Gjerningerne ere , jo mindre tro vi ogsaa . Gjerningernes Urenhed , og at de ere saa simple og ringe , hindrer os fra Troen ; ellers vilde vi over al Maade prise og forherlige Guds Naade og Barmhjertighet ) i disfe simple og ringe Gjerninger , og vor Tro vilde derved saare styrkes . Thi saa stor Omsorg og Bekymring nerer Gud for os , at han ikke engang kan glemme et Håar , ikke en Tåare , ikke en liden Og det mener den Hellig-Aand , nåar han vandrer saa lavt nede , idet han beskriver sine Hellige , at endog deres allerringeste Gjerninger tekkes Gud vel . Et Christenmenneste er dog kosteligt for Gud ; han har intet faa Ringe , udeu at det jo tekkes Gud . At udgyde sit Blod , do , svede , stride og kjempe mod Djevelen er i Sandhed en stor Ting og tekkes Gud saare vel ; men du maa stutte saa : er du kun troende , saa sinder Gud Behag ogsaa i de naturlige , kjodelige og legemlige Gjerninger , enten du spiser eller drikker , enten du vaager eller sover ; hvilket jo Altsammen er legemlige og naturlige Gjerninger . Saa stor en Ting er Troen . 35. Derfor fe allerfsrst til , at du bliver en Christen , og at Personen bliver Gud velbehagelig ved Ordet , ved Daaben og Sakramenterne. Naar Personen er troende og henger ved Ordet , ikke forfolger det , men takker Gud derfor , da stal du ikke gjore Andet , end hvad Salomo siger Predik . 9 , 7. 8. 9. 10. : oed dit Brod med Gllrde , og drit din Vin med et godt Mod ; thi Gud har allerede Behagelighet » til dine Gjerninger . Lad dine Klceder altid verre hvide , og lad Olie ikke fattes paa dit Hoved . Se til , du lever med en <b>Hustru</b> , forn du elster , alle din Forfcengeligheds Livsdage , som Gud har givet dig under Solen , alle din Forsamgeligheds Dage ; thi det er din Del i Livet og i dit Arbeide , som du arbeider uuder Soleu . Alt det , som tommer for din

<h4>1330</h4>53. Her ser du , at disse Ting ere ganste og aldeles beskrevne paa den samme Maade , som man pleier at beskrive Hyrders Anliggender , og hvorledes de tale med hinanden . Og den Hellig-Aand bruger dog saa mange Ord derom , saaat en verdsligsindet Leser snart faner Modbydelighed og Kjede derfor . Andre Beretninger om store , vigtige Ting , fnnfom Sodomns og Gomorrns Odeleggelse og Isaks Offring , har han afferdiget med faa Ord , neppe i fem eller sex Vers ; men nåar han kommer til disse urene , kjodelige og taabelige Ting , bruger han saa over al Maade mange Ord derom , nemlig foråt vi stulle vide , at Herren har Behagelighet » til dem , som ham frygte . Thi ere vi troende og sikkre paa Guds Naade , som han har vist os for Intet , faa maa vi ikke tvivle paa , at jo Alt , hvad vi gjore , behager Gud saare vel , og at han endog har talt Haarene paa vort Hoved ; ja , at han har Velbehag i , at Jakob kysser og favner Rachel og velter Stenen fra , hvilket Altsammen fortelles paa dette Sted som saure hoie og kostelige Gjerninger for Guds og de hellige Engles Oine . Gud har ikke kunnet glemme dem , men har villet , at de stulde opflrives os til <b>Lerdom</b> og Trost . 54. Om vi nu end plages haardelig i dette Liv og ere underkllstede megen Ulykke og Jammer , endelig ogsaa Doden selv , saa stulle vi dog vide , at det Altsammen er lutter kostelige Ting , som der staaer Ps . 116 , 15. : det er tosteligt sor Herrens Dine , unar hans Hellige ds. Dette stulle vi ogsaa forståa om andre Gjerninger , enten vi spise eller drikke , sove eller vange , som St . Paulus siger Col . 3 , 17. : Alt , hvad I gjore i Ord eller i Handling , det gjorer Alt i den Herres Jesu Navn , takkende Gud og Faderen formedelst ham . Og i saadanne ringe Gjerninger stulle vi ove os , vaadet vi efterhaanden kunne gaa fremad og venne os til hoiere Ting . Men Troen , som virker store og ridderlige Ting , nedlader sig ogsaa til de huslige og ringe Gjerninger ; den rogter Fnnrehjorden og vånder den , den melker Gjederne . Thi Gud hnr stnbt bande Dnnren og den <b>Vise</b> , den Lille og den Store , det Kostelige og det Ringe . Derfor hnr hnn ligesnnmeget Velbehag i Tjenernes eller <b>Pigens</b> Gjerning som i Herrens , ligesnnmeget i Hustruens som i Mnndens . Ver kun troende , og hold for alle Ting Gud i 3 Ere i de Ting , som angaa Kirken og Religionen ; sorg for , at du forbliver ved Guds Ord . Dersom I blive i mig , siger Christus Joh . 15 , 7. 8. 10. , og mine Ord blive i Eder , da stulle I veere megen Frugt o . s . v.

<h4>1350</h4>67. Derfor fommer det sig jo ikke , at en saadan hellig Mant » stal vere saa taabelig og sandselig , at han tager Hensyn til en <b>Piges</b> Skjsnhed og mere elster en stjon <b>Pige</b> end en , som er heslig og ikke stjon . Thi man kan ikke sige , at han gjorde det af den Grund , at han gjerne vilde , hun stulde fode ham Born ; da jo de <b>Kvinder</b> , forn ikke ere stjonne , i Almindelighed pleie at faa flest Born , som man ogsaa kan se ved Lea , og dog kunde hun ikke ved nogen Fodsel opnaa , at Jakob stulde elske hende saa inderlig som Rachel . Derfor kan man ikke undstylde ham under det Paastud , at han gjorde det af Hensyn til Born ; men det er en ringe , skjendig og naragtig Ting , nemlig at elske en <b>Pige</b> , fordi hun er stjon , og ikke fornemlig i Haab om Born . 68. Det stulde vel billig ste , at man elskede det <b>kvindelige</b> Kjon i sig selv , blot af Henfyn til Bornene , for hvis Skyld dette Kjon er flabt af Gud , og ikke benyttede det til Vellyst og Utugt . Thi at <b>Kvinderne</b> ere stabte til at sode Born , viser bele deres Legemsbygning . Men der er Faa , som tåge Hensyn dertil , og Jakob overser ogsaa dette Oiemed . Thi han elster Rachel , fordi hun er stjon ; Lea derimod elster han ikke , fordi hun ikke var faa stjon , uagtet det dog mindre fommer sig for gamle Ment » end for Inglinge at tåge faadant Henfyn til Skjonheden. 69. Men dette stal man lere og indprente i Kirken , foråt vi ikke stulle vere bange for HEgtestanden , saaledes som Papisterne have skjendet og fordomt den med sine Narrestreger . Thi derfor skriver den Hellig- <b>Aand</b> dette , fordi han derved vil <b>vise</b> , at Gud heller ikke vil forkaste og fordomme dem , som ved sine Wgtehustruer tåge Hensyn til Skjonhed , og at et saadant Valg kan ste udeu Synd . Derfor maa Ingen tro , at han fynder , fordi om han mere elster en stjon <b>Kvinde</b> end en heslig . Saaledes tunne vi deraf hente <b>Lerdom</b> til vor Forbedring . 70. Dernest havde Jakob det Haab , at han stulde faa Born med den <b>Pige</b> , forn var stjon og af et sterkt Legeme . Thi nåar man indgacter KEgtestab , stulde man med Rette legge Merke til , at Sterke giftede sig med Sterke , Raste med Raste . Denne Forskjel iagttages jo med de ufornuftige Dyr ; men blandt Menneskene ster det ikke saaledes , paa Grund af den fordervede Natur , som er kommen af Arvesynden . Thi deraf kommer det , at Nogle fodes sterke , Nogle svage , og undertiden Nogle ogsaa spedalske , Nogle blinde , Nogle taabelige , hvilket er Foreldrenes Skyld . Thi for Arvesyndens Skyld maa man sinde sig i , at sterke og svage Personer sammenfoles i AEgtestab , fordi der maa randes Bod paa denne Arvesyndens Skade med den Hjelp , som Gud har vist os i HEgtestabet , hvordctnne nu end de Folk ere , som deri forenes .

<h4>2317</h4>over , at de vare forkastede og fordomte . Thi de bleve plagede af Djevelen med Vefpottelsens <b>Aand</b> . 149. Saa vergede nu Nonuen sig imod denue Anfeglelse alene med de Ord , at hun fagde : jeg er en Christen ; det er , jeg er dobt i Guds Sons Blod , jeg er bespist med Christi Legeme og Blod , derved holder jeg mig fast , med den Trost er jeg tilfreds , om ogfaa Gud felv vilde sige Andet . Ligesom Abraham blev befalet at drebe sin Son , i hvem Forjettelsen var given ham , og dog alligevel troede paa det Fasteste , at Gud vilde holde ham sin Forjettelse . 156. Dette maa man flittig drive paa for deres Skyld , som herefter stulle blive Prester . Thi der vil altid vere Nogle , som have denne Anfeglelse , og for at da disfe kunne blive opreiste og styrkede ved Presterne . Som paa den Maade , at de sige : ver trostig , min Son , tro , at du er dobt , at du er neret og bespist med Herrens Nadvere , dertil har du faaet dine Synders Forladelse med Paaleggelse Me af mine men Guds Hender , forn fagde til dig : jeg tilgiver dig dine Synder og forjetter dig det evige Liv . Naar de fatte og antage dette med fast Tro , faa ville Aandens Anfegtelfer og Bespottelser forsvinde . 151. Thi Abraham solte uden Tvivl ogsaa stor Skrek og Beven i sit Hjerte , da han sik Befaling til at drebe sin Son . Men imidlertid beholdt han denne Tro , hvori han vist sluttede , at om haus Son end blev til Aste , saa vilde han dog blive Fader til mange Arvinger og Efterkommere , efter den guddomntelige Forjettelse . Hvorledes , bliver da Gud en Logner ; holder han da ikke sine Forjettelser ? Saadanne Tanker faldt ham ind . Og dog adlyder han alligevel Befalingen og tenker : ligefom Herren har ladet denne min Son komme af en gammel , ufrugtbar <b>Moder</b> og af mine Lender , faaledes kan han ogfaa opvekke ham igjen , om han end ovbrendes og bliver til Aste . Paa denne Maade , siger jeg , kjempede ogsaa Abraham med Gud , hvilket er en af de allerstorste og tungeste Kampe . Men Gud viser deri sin fuldkomne og gode Villie . 152. Hvorvel nu denne Kamp Me kan forstaaes eller taales uden af de Hellige , saa maa man dog have denne <b>Lerdom</b> og Trost til at styrke os med , vaadet vi ikke skulle ovsluges af Djevelen . ; dog Gud er trofast , forn ikte frister os over Formue , 1 Cor . 10 , 13. Thi det lere vi af dette Jakobs Exempel , der var faare svag til denue Kamp og dog alligevel Me beseires . Men Gud berer sig saaledes ad mod ham , at han ikke kan erkjende , at det er Gud , som kjemper mod ham ; han mener , at det kun er en Engel . Men det er Gud , som berer sig ad som hans Fiende , retsom om han vilde drebe ham og berove ham Forjettelsen og Velsignelsen og give den til hans Broder Esau . Og Ingeu kan med Ord udsige , hvad han havde for Tanker . Saa tenkte han uden Tvivl : hvad er jeg dog for et usfelt , eleudigt Menneske ! Er jeg da blot stabt til bestandig at have Ulykke ? Maa jeg alene da bestandig have den ene Ulykke ovenpaa den anden og plages saaledes dermed , at jeg aldrig kan komme til Ro ? Der er dog intet ulykkeligere Menneste paa Jorden end jeg . Jeg ser , at min Broder Esau hersker , iriumpherer , tager til og bliver stor med stor Herlighed , med meget Gods , med Born , Borneborn og store Indtegter . End om vor Herre Gnd var bleven anderledes tilsinds , havde forkastet mig og tåget min Broder til Naade ?

<h4>2972</h4>vife , hvad Efau havde for Krigsfolk , da han drog Jakob imode . Men her opstaaer et Sporgsmaal : hvorfor kaldes Oholibama Anas Datter og Zibeons Datter ? Hvorledes kan hun vere to Mends Datter ? loderne drive et underligt Spil dermed , og have to skjendige Meninger derom : den forste er , at Zibeon var en ugudelig Maud og Wgtestabsbryder, og at han derfor sov hos sin Fader Seirs <b>Hustru</b> og avlede Ana , saaledes at Ana var Zibeons naturlige Son , men i Forhold til Seir blev holdt for at vere hans Broder . lodernes anden Mening er , at Zibeon besov sin Son Anas <b>Hustru</b> , saaat han altscta stal have drevet Blodstam baade med sin <b>Moder</b> og sin Sonnekone . Saaledes anse de Oholibama for avlet i Blodskam , idet hendes Fader var avlet i Blodstam . 28. Men jeg hader af ganste Hjerte loderne og deres Rabbineres Kommentarer eller Udleggelser . Thi dette er deres <b>Vis</b> , ja , det er en rent aabenbar Galstab hos dem , at de , nåar de blot paa nogen Maade kunne , faa ud af Texten sine Egnes Pris og Hedningernes Skjendsel . De ere forbandede Folk , der holdes fangne og ere besatte af Djevelen , og den Herre Christi Billede eller Lignelse om Bjelken og Skjeven , Lue . 6 , 41. , passer godt paa dem , ligeledes ogsaa hvad han siger Malth . 23 , 24. om dem , som afsie Myggen , men nedsluge Kamelen . Thi hvad den ene Phariseer praler af : jeg er Me forn andre Mennester, Lue . 18 , 11. , det prale hel uforskammet ogsaa loderue af og sige : ' vi ere Guds Folk , prydede af Gud med store ' Gauer . Og en af deres Rabbinere har sagt : det er et Under , om man blandt alle Hedninger kan finde een viis Mand . I Israels Folk derimod stal der vere lutter fortreffelige , <b>vise</b> Mennesker . 29. Ja , det er et saare stjendigt og forderveligt Folk , og det var det allerede i Fedrenes Tid under Jakob og David . Thi Kjere , se dog , hvad for herlige Bedrifter de have udfort . Ruben besteg Bilhas , sin Faders Hustrues , Leie ; luda driver Blodskam med sitt Sons <b>Hustru</b> . En saadan Klik sette disse Hyklere ogsaa paa Ana , Oholibamas Fader . Men hvorfor se de ikke forst Bjelken i sit eget Oie ? Thi hvad er skjendigere , end Absalons Opror mod sin Fader David , hvorved han jager ham ud af Landet og , saameget det staaer til ham , dreber ham og skjender ham hans Medhustruer ? Lad nu Rabbinerne gaa hen og prale af lodernes <b>Visdom</b> . Desuden have de altid veret et oprorst Folk mod sine Propheter og Konger ; mod Herren knurrede de , Propheterne drebte de paa det Grueligste . Derfor hader jeg Rabbiuerues Kommentarer eller Udleggelser , i hvilke de ilde forfalske den hellige Skrift med sine Logne . 36. Af denne Text , hvori Oholibama kaldes Anas Datter og Zibeons Datter , ville de flutte , at hun havde to Fedre , og at Ana stal have besovet sin <b>Moder</b> 0. s . v. Dette er en rigtig jodist , det er en eselsagtig Folgeslutning , der ogsaa forkastes af Lyra og af Pagnintts^). At der nu fandtes saadanne Laster hos Hedningerne , derom tvivler Ingen ; men om Ana har man ingen klar Text eller intet Vtdnesbyrd af Skriften , hvorved han kan overbevises derom , saaledes som man har klare Beretninger om Ruben og luda , ligeledes ogsaa om de andre Brodre , der solgte Joseph og drebte sin gamle Fader ved Knmmer og Hjertesorg . Thi saaledes begreder hatt sin Son , som han havde

<h4>3083</h4>39. Jeg undrer mig over , at de saa lenge bleve tactlte i Jakobs hellige Menighed . Jeg havde fandelig ombragt dem Allesammen eller stodt dem ud af Hufet . Thi , du kjere Gud , hvad er dog det for en stor Afsindighed og Ondstab at vredes paa en saadan Fader , fordi han elster sin Son , der er from og gudfrygtig , dertil ogfaa fodt af hans kjereste <b>Hustru</b> ? Derfor have vi ogsaa sagt , at man maa ' gjore denne Ondstab stor og svar , saameget det blot er muligt ; thi de gjore det dog altfor galt . 46. Forresten opstaaer her en Tvist mellem de Sproglerde , nemlig om Betydningen af den Grund , som Moses angiver for Jakobs Kjerlighed til Joseph , nemlig at han var bleven avlet og fodt i Jakobs Alderdom . Thi man kan med mange Beviser godtgjore det Modsatte , og Historien vidner ogfaa noget Andet . Burgensis beretter , at der i den chaldeiste Text stal staa , at Jakob elstede Joseph , fordi han var faa <b>vis</b> og klog , at han vel kunde overgaa enhver Alder i <b>Visdom</b> ; og her kunde man da anfore det Sted Ef . 65 , 20. : den Dreng , forn er bleven hundrede Aar gammel , stal do , det er : de stulle Alle blive <b>vise</b> over sin Alder ; Drengeue skulle vere saa <b>vise</b> , som om de vare hundrede Aar gamle . 41. Lyra strider herimod og vil , at Joseph af den Grund stal kaldes hans Alderdoms Son , fordi hans Fader havde avlet ham i sin Alderdom . Men jeg kan ikke forståa , hvorledes deune Mening kan forliges med , hvad der sidenefter siges om Benjamin , som dog fodtes nesten tretten Aar efter Joseph , og han kaldes da ogsaa af sin Broder luda en Alderdommens Son fremfor lofeph . 42. Hvorvel jeg nu pleier at overlade faadanne Kivsmaal til de Sproglerde , saa synes dog Lyras Forklaring at vere daarlig , aldenstund jo Benjamin er tretten Aar yngre ; Joseph derimod er den syvende Son , efter Dan og Navhthali ; Gad og Asser ere fodte enten efter Joseph eller samtidig med ham . Tilsidst fodtes Gad , Asfer , Ifaschar , Sebulon og Diuct ; alle disse ere yngre end Joseph . Hvorledes stal da mt Texten forstaaes , at Joseph stal vere fodt i sin Faders Alderdom ? Historiens Rekkefolge vifer jo meget mere , at dette ligefaa godt kan siges om alle Jakobs Sonner , aldenstund Jakob jo forst fik Born , da han var 74 Aar gammel . Derfor vil jeg overlade de Sproglerde at lose dette Sporgsmaal ; thi jeg ved , at det hebraiste Sprog har mange Maader at udtrytke sig paa , som nu nesten ere os übekjendte , og

<h4>3145</h4>faa for Ende . Han fer , at Folks Planer og Beslutninger for have faaet dette eller hint Udfald , og at der ikke er noget Tegn til , at Gud stulde lade nogen Forhindring indtrede , hvorvaa den fattede Plan kunde strande . Derfor falder det ham let at spaa om tilkommende Ting . 94. Saaledes spaaede ogsaa Lichlenberger for Bayrerne . Thi Djevelen faa vel , hvad dengang Keiser Maximilians , Pfalzgrevens og de andre Fyrsters Planer gik nd paa . Dette gjor han ogsaa ellers , og hvis Gud ikte setter sig derimod efter sit synderlige Raad , saa ster og opfyldes det vel ogsaa , som saaledes er forudforkyndt . Men det bryder Djevelen sig ikke om , om en faadan Drom ogsaa er Menneskene nyttig i deres Anliggender eller ikke ; ja om saa end Alt styrter sammen , er han lige glad derved . 95. Men forresten er hele dette Liv , hvori vi leve , kun en Drom og ligesom en duukel Nat . Thi ellers lyser der i Alverden intet andet Lys end dette levende og legemlige Lys . Solens Lys er os ligesom en Anledning til Vildfarelse , idet vi mene , vi ere i Lyset ; men i Virkeligheden famle vi os frem ligesom ien msrk Nat . Derfor , siger jeg , er dette Liv i Virkeligheden en Sovn og <b>aandelig</b> intet Andet end en Nat , ligesom i losephs Drom hans Foreldre sammenlignes med Sol og Manne , men hans Brodre med Stjerner , som stinne i dette Livs Morke . Det Samme vidner ogsaa Moses , nåar han siger Ps . 90 , 5. : du bortstyllede dem ; de ere som en Sovn ; det er , vort Liv er kort , flygter fnart hen ligesom en Elv eller stor Flod strotnmer afsted , ligesom et Vand bruser forbi . Saaledes , siger Moses , har du gjort dem ligesom en rindende og brusende Strom . Saaledes siger ogsaa Job Kap . 14 , 1. 2. : et Menneste , fodt af en <b>Kvinde</b> , lever en stakket Tid og mettes af Uro . Han ganer op forn en Blomst og asstjeres ; han flyer forn en Stygge og bestllller ilte . Og Moses tilfoier , hvad der er endnu mere : de ere forn en Sovn ; det er , vort Liv her paa Jorden er imod det evige Liv intet Andet end ligesom en Sovn . 96. Saaledes henge og sveve vi i dette Liv ligesom midt i en Nat og det tykkeste Morke , hvis Me . den rette Morgenstjerne ovgaaer og skinner for os , hvorom Pf . 119 , 105. siger : « herre , dit Ord er en Lttgte for min Fod og et Lys paa min Sti . Thi have vi Me den hellige Skrift til at lyse for os og styre alle vore Gjerninger , saa er hele dette Liv og alt Fornuftens Lys , ja al <b>Visdom</b> og alle Menneskers Raad dertil Me Andet end idel Morke og lutter Forvirring . 97. Augustus , Alexander og Julius Cesar befindesig ligesom i en tyt Tange , vide selv ikke , hvad de gjore . Saaledes ogsaa Nebukadnezar, Pharao og alle andre Konger og Herskere ; dog indbilde de sigimidlertid, at de udrette Meget og ere overmaade <b>vise</b> . Men det er lutter Dromme , som der staner Es . 29 , 7. : alle Hedningernes Mangfoldighet,, forn stride imod Ariel , stal vere forn en Drom og som et Nattcfyn . Saaledes ere ogsaa alle store Heltes og Herskeres Historie , som prises af Digterne og ellers i Hedningernes Aarboger , saasom Hektor og Achilles , blot som Dromme ; thi som nåar den Hungrige drommer , og se , han eder , men nåar han ovvaagner , da er hans <b>Sjel</b> tom , og ligesom nåar den Tsrstige drommer , og se , han dritter , og han ovvllllgner , og se , han er tret , og hans <b>Sjel</b> er torstig , saaledes stal alle Heduiugernes Mangfoldighet » blive , som stride imod Zions Bjerg . Thi det hender ofte , at En i Dromme tykkes sig at sinde en Sek fuld

<h4>3241</h4>var tolv Aar gammelt , ogsaa uttddrog sig sine Foreldre , Lue . 2 , 43. Saaledes forholder han sig endnu mod de Gudfrygtige og Fromme , faaat det fer ud / forn om han flet ikke kjendte os ; selv om han i Forveien har givet os store Forjettelser , og efterat han visselig har ovrettet sin Pagt med os , lader han os dog omkomme saaledes , som om han ganste havde glemt sin Forjettelse , slet ikke agtede paa os og ganste havde forkastet os . 184. Da er det nu de Christnes <b>Visdom</b> , at holde stille for dette guddommelige Naad og holde ud i Troen paa den Forjettelse , forn er os given . Thi denne Forjettelse er jo sikker og <b>vis</b> nok , og Herrens Pagt staaer ogsaa fast , forn Ps . 121 , 4. siger : hnn stal Me slumre og ei sove , han , som bevarer Israel . 185. Men derpaa svarer den menneskelige Fornttft saa : alt dette er vel herlig og smukt sagt ; men jeg erfarer og fornemmer alligevel det Modsatte . Thi han fover ikke alene , men fnorker vel endog dertil ; ja , der er intetsteds nogen Gud , som sorger for os eller agter paa os . Saaledes er nu Jakob <b>vis</b> paa den guddommelige Forjettelse , ved ogsaa , at Guds Pagt vil staa fast ; og dog omgaaes Gud saaledes med ham , at det ser ud , som om hverken han selv eller hans Son Joseph har de hellige Engle til Vogtere og til at modstaa hans Brodres grumme Vrede ; alle tie de bttmstille , lade Djevelen storme los paa den hellige Kirke og Menighed , rase og larme mod deu . Hvor er da nu Gud henne ? 186. Derfor paamiudes vi nu ofte om dette , paadet vi as dette Exempel stulle lere , at man flet ikke stal tvivle paa Gud , uaar han forjelter Noget . Thi ligesom Gud ikke kan lyve , saaledes kan han heller ikke Andet end sorge for os , fornemlig nåar vi holde fast ved hans Forjettelse . Thi nåar denne gribes fast , er det umuligt , at vi skulde forlades , fordi jo Gud er fattddru . Hvis hau derfor tilsteder , at vi anfegtes , fores ned i Helvede eller drebes , forn vi se i dette Jakobs Exempel , stulle vi altid fly hen til hans Forjettelse og ganste lukke Oinene for den strekkelige Forargelse , hvormed vi saaledes korsfestes og plages . 187. Jakob og hans Son lofeph proves og befveres . fandelig saare tungt , hvilket var ganske og aldeles imod den guddommelige Forjettelse , og det ser ikke miudre ud til Noget , end til at Gud stulde agte paa dem og sorge for dem . Thi han fender dem jo ingen Engel , ja ikke engang et Stråa , hvorved Djevelen kunde opholdes og drives tilbage ; men lukker Dor og Vindu op for ham , foråt han ret kan rafe paa det Grummeste baade mod Fader og Sov . 188. Jakob har vistnok en saare stor og rig Forjettelse , som ogsaa allerede var gaaet i Opfyldelse , den Velsignelse nemlig , at han havde faaet tolv Sonner . Men alligevel maa han endnu bestandig kjempe med mange store Besverligheder , som mode ham , som om han ingen Gud eller ingen Forjettelse havde . Dog maa vi flutte og alligevel vere visse paa , at hau ingenlunde var forladt af Gud . Thi den Forjettelse staaer endnu altid fast Es . 49 , 15. 16. : kan og en <b>Kvinde</b> glemme sit diende Varn , at hun Me forbarmer fig over fit Livs Son ? ja , om Nogen af dem tan glemme dem , da vil jeg dog Me glemme dig ; fe , jeg tegnede dig paa begge Hender . 189. Men hvorfor <b>moder</b> der os alligevel saa megen forferdelig Kummer fra vort eget Kjod , vore Born og vor Slegt ? Svar : det er

<h4>3439</h4>nu i denne Verdens sidste Tid . Saaledes holder jeg for , at det var en fynderlig Guds Gjerning , at luda i deune Alder var istaud til at avle Born . Nutildags er der saare faa , som i sit attende Aar ere istand til at gifte sig og faa Born . Men Patriarkerne kom i faa Aar til den fornodne Legemstraft , saaat de endog avlede Born , forent , de vare rigtig voxne . 22. Men efterfom dette Kapitel intet Andet indeholder end Fortellingen om , at luda avlede Born og drog bort fra sine Brodre , og desuden ogsaa den skjendige Blodstam , som han begik med Thamar , sin Svigerdatter , opstaaer her atter et almindeligt Sporgsmaal , nemlig hvorfor dog den Hellig-Aand har ladet disse saa skjendige Ting , som neppe kunne tales om , ovstrive og ovbevare , foråt de stulle fortelles og lefes i Kirken ? Thi han synes ikke at have gjort ret deri , at hatt har udeladt attdre storre Ting og fortalt disse , som ikke alene ikke ere nodvendige, men Me engang verdige til at bringes for Lyfet og beholdes i Erindringen . Thi hvem vil tro , at saadanne Ting kunne leres til Salighed eller til Kirkens Forbedring ? Han forteller , hvorledes luda veg bort fra sine Brodre , tog en fremmed <b>Kvinde</b> til Wgte og derpaa besmittede sin Svigerdatter med Blodstam ; det havde jo dog veret bedre at skjule dette og forglemme det i al Evighet » . 23. Canonisterne og Papisterne kumte lettelig flippe bort fra dette Sporgsmaal , idet de sige , at det gamle Testamente er en Bog , som kun handler om lodernes Samleier og Horeri . Men de legge derved sin Uforstandighed for Dagen og vidne nokfom , at de ere Wster og grove Svin , idet . de tale om saadanne Ting , hvorvaa de dog intet Skjott have og hvorom de slet Intet vide , og de forhaane dermed ogsaa dett Hellig- <b>Aand</b> , som om han Me skulde vide , hvad man stulde skrive eller foreholde Guds Kirke og Menighed . Hieronymus har svaret paa dette Sporgsmaal ; men Me det alene er Aarsagen , som han , anforer , nemlig nt Christus har villet prise sit Slegtregister fra Forst af i den hellige Skrift , og at , ' da Thamar var hans Stammoder , stttlde hendes Navn vere bekjendt for Hvermand . Men desuden foreholdes os her en stor Trost , idet nemlig de hellige Patriarker og disse herlige Fedres Born , luda og de Andre , beskrives som saadanne Mennesker , som have veret sulde af Svaghed og fulde af store Pletter eller Synder , hvormed deune stakkels Natur er besveret , og at Gud har ledet og fort dem underlig ved sin Hellig-Aand , dog saaledes , at han tilsteder , at deres medfodte Tilboielighed , det er Synden og Arvesyndens Frugt rorte sig og brod frem hos dem . Thi det var jo en strekkelig Synd af dem Begge , baade af luda og Thamar ; om man end kunde anse den for noget übetydelig , kan man dog ingenlunde ganste undstylde den . 24. Men disse Exempler fortelles os til <b>Lerdom</b> og Trost og ogfaa til vor Troes Styrkelse , paadet vi derved stulle betragte Guds store Naade og Barmhjertighed , der ikke alene gjor de Fromme og Retferdige salige , saasom Abraham , Isak og Jakob , men ogsaa de Uretferdige , faasom luda , Thamar , Ruben , Simeon , Levi , hvilke vare saare store Syndere , saaat Ingen maa vere formastelig eller stolt for sin Retferdigheds eller Visdoms Skyld , men heller ikke Nogen stal forsage over sine Synder . 25. Saaledes prises nu i den hellige Skrift de kjere Fedres stjonne Exempler , Abrahams , Ifaks og Jakobs , der vare ligesom Lys for Alverden og i Guds Kirke eller Menighed ligefom Stjerner , Solen

<h4>3986</h4>syv blive tilovers , eller man kunde tyde det om syv rige og syv fattige og elendige Slegter . 64. Dette fer vel ud som <b>Visdom</b> og synes ikke at vere en ganste urimelig Mening ; men den er endnu meget langt fra den rette provhetiste Mening . Thi det kunde Ingen sige forvist , at de syv Kjor stulde betyde ligesaamctnge billige og gode eller dyre og stette Aar . Men da denne Udtydning forst er givet med sin Udgang og Opfyldelse , synes den saa simpel og klar , at Enhver synes at kunne have forstaaet det . Du kunde vel finde Mange , som indbildte sig og mente , at de ogsaa lettelig og uden al Moie stulde have forstaaet det . Saaledes tilskrive Sektaanderne og Svermerne nutildags sig selv , hvad de have hort af os , og ville nok have hoi Pris og Berommelse for , at de nu kunne dadle og foragte Andres Arbeide . 65. Men der er ingen Tvivl om , at denne Udtydning vel vil have kunnet give de Alleroverste og Lerdeste i Riget nok at bestille ; de arbeidede og forsogte sig Allesammen forgjeves derpaa . Thi Kongers og Menighedsoversters Dromme ere Svaadomme , som kreve den Hellig- <b>Aand</b> for at udlegges ; for Andre ere de ganste lukkede og stjulte . Saaledes ere Spaadommene hos Efaias og andre Propheter om Christi Rige , at han stal underlegge sig al Verden , vistnok klare ; men dog forstaaes de Me lettelig af Hvermand . Thi loderne dromme om , at han stal bruge Sverdet og legemlig Magt dertil . Saadanne Propheter vare forseglede og forborgne fctctlettge , indtil Christus og Avostlerne ctabnede alle Segl og lerte , at det skulde vere et acmdeligt Rige , som stulde forlose os stattels Mennester fra den evige Dod og berige os med himmelste og evige Goder . Men for loderne er det fremdeles lukket og stjult , Alt som Propheterne have forudforkyndt baade om Christus og hans <b>Moder</b> , som der staaer Es . 29 , 11. : alt deres Syn stal vere Eder som en beseglet Bogs Ord . 66. Saaledes har Joseph en prophetist Aano , derfor fvctrer han ogsaa Pharao rettelig og siger : Pharaos Drom , den er een , uagtet han dog havde to Syn , nemlig med Kjor og med Ax . Men havde Fornnften veret noen den Hellig-Aand , faa havde den nodvendig domt , at det Me var een Drom , hvorvel nu ogfaa Hvermand fynes , at det er een , efterat det er aabenbaret af Gud .

<h4>4179</h4>258. Paa den Tid , da Joseph udtydede Drommene og tog sig en <b>Hustru</b> , var han tredive Aar gammel , hvilket Moses forteller foråt man deraf kan flutte , hvor lenge han havde veret i Fengsel og Treldom . I sin Alders 17 de Aar blev han solgt , i sit tredivte Aar blev han hevet til kongelig Herlighed . Saaledes tjente han sin Herre trofast og godt i ti Aar og forbedrede hans Hus , forogede hans Gods og Eiendom , faaat han blev rig . Men i tre eller fire Aar var han i Fengsel , da han var bleven anklaget for Horeri af Horkvinden . 259. Dette er strevet os til <b>Lerdom</b> , paadet vi stulle lere at forståa den store Jammer og Elendighed , som Joseph led , hvorom Ps . 105 , 18. siger : de plagede hans Fodder med lerubolte , han selv tom til at gaa i Jern . Hvad enten du nu vil forståa Jernet saaledes , at hans Legeme var bundet med Lenker , eller i den Mening , som Simeon siger til Maria Lue . 2 , 35. : ogsaa din egen <b>Sjel</b> stal et Sverd gjennemtrenge , saa er det dog itte usanosynligt , at han i sin Herres sorste Vrede og Raseri blev lagt i Bolt og Jern ; men sidenefter sandt han Naade hos Fangevogteren , der loste hans Lenker og lod ham have Opsynet med Fangerne . 269. Dette Exempel foreholdes os , paadet ogfaa vi stulle venue os til Taalmodighed , dertil ogfaa til Haab og Standhaftighet » under Kors og Ulykker , om end vore Hjerter haardelig plages og pines med den lange Opsettelse , forent » Hjelpen kommer . Thi Gud fer Enden paa vore Trengsler , forn han siger til Israels Born Jer . 29 , 10. 11. : efterat sytti Aar ere sulde i Babel , da vil jeg besoge Eder , og jeg vil stadfeste mit det gode Ord over Eder , at fore Eder tilbage til dette Sted . Thi jeg ved de Tanter , forn jeg har tenkt over Eder , siger Herren , nemlig Fredstanker , og Me til Mytte , at give Eder Velonuing paa det Sidste og Eders Forhaabning . Fcmgenstctbet i Babel stal ikke vare evig ; det er sytti Aar om at gjore ; men det gjor Ondt at have Taalmodighed og vente paa Enden . 261. Derfor skrives nu alt dette , foråt vi stulle lere og betragte det og lere at bere og taale Guds Haand , der hjemsoger os og tngter os faderlig . Thi Straffens eller Trengselens Tid er ikke uendelig , som

<h4>4239</h4>det . Dernest er det Ord at bryde eller sonderbryde ogsaa bleven anvendt om <b>Sjelen</b> , saaledes en fsnderbrudt Aano , det er en bedrovet , elendig Aano , der vakler mellem mange forskjellige Tanker og Planer og Me ved hverken ud eller ind . 4. Derfor . stal man merke sig og holde fast ved denne Maade at tale paa , nemlig at hos Hebreerne det at bryde Brsd er det Samme som at uddele det eller at kjobe og selge Brod eller Mad , hvilket er en billedlig Tale . Men hos Propheterne staaer det oftere end paa andre Steder i Skriften . Salomo har ogsaa brugt denne Maade at tale paa Ordspr . 11 , 26. , hvor han siger : Velsignelsen kommer paa hans Hoved , som selger Kornet . Thi Hebreerne bruge meget at tale i Billeder , hvilket ogsaa er brugeligt i andre Sprog ; ja , de synes mere end Andre at have havt Lyst til saadan billedlig Tale . 5. Dernest er ogsaa det at merke , at Hebreerne Me have at vere som selvstendigt Verbum , men istedetfor at sige : jeg er Herren din Gud , sige : jeg Herren din Gud . Derfor bruges ogsaa det hebraiste Ord lesch paa dette Sted Me covulativt eller som et blot Bindeord , men det betyder : der er forhaauden , der findes . Deraf kommer Ordet Thufchijah i Ordfpr . 8 , 14. , hvor <b>Visdommen</b> siger om sig selv : Raad og Thufchijah det Bestandige hore mig til , jeg er Forstand , mig horer Styrke til ; mit er baade Ord og Gjerning ; mig tilkommer baade Naad og Daad , det er , hvad jeg rander , det udforer jeg ogsaa . Thi saameget betyder Thufchijah , nemlig at sette enhver Ting paa sin Plads . Ellers er det vel ogsaa et almindeligt Ord , som betyder en Grund , <b>Visdom</b> eller men egentlig betyder det , at de Ting , som man bestemmer , suldbyrdes og bringes til Udovelse . F . Ex . : Gud giver os Raad i sit forkyndte Ord ; uaar vi holde fast ved dette Ord , faa folger ogsaa Thufchijah derefter , at nemlig det , som han forjetter os , ogsaa visselig gives os og ikke staaer Feil . Saaledes er det ogsaa paa dette Sted Me blot Ord eller et usikkert Rygte om Kornets Brydning eller tiddeling, men det er virkelig Sandhed : i Egypten er der virkelig fuldtov af Korn og Levnetsmidler . 6. Derfor siger nu Jakob til sine Sonner : hvorfor sidde I og se paa hinanden ? Moses tilkjendegiver herved , at Jakobs Sonner rctctdsloge og lagde Planer med hinanden om , hvad der kunde vere at gjore , efterfom Hungeren var svar . Du Ruben , du luda , sagde » de vel , hvad vil du ractde os til , sorat vi kunne finde Hjelp i denne Hungersnod og frelse baade vort eget og vore Familiers Liv ? Medens de nu saaledes ractdfloge og saa paa hinanden , sagde Faderen : ak hvorfor sidde I saaledes og se paa hinanden ? Imedens I saaledes forgjeves holde Raad , maa vi do af Hunger . Se , jeg har hort , at der i Egypten virkelig er Korn nok til Uddeling og Salg ; drager ned og kjober os Korn ; ser til , at ogsaa I kunne faa vere med ved hin Uddeling. 7. Men Mofes viser , at disse elleve Patriarker Allesammen boede Med sin Fader Jakob i Hebron , uagtet de dog hver for sig vare Husfedre, havde Born og Borneborn , Tjenere og <b>Piger</b> . Derfor er det et Under , hvorledes dog saa mange Familier , der hver bestode af saa mange Personer , kunde bo Alle sammen iet fremmed Land . Men det er min Tro , at Lea og de andre gamle <b>Kvinder</b> i Jakobs Familie dengang vare dode , og at der var kommen andre , unge <b>Kvinder</b> i deres Sted , nemlig hans elleve Sonners Hustruer , dertil ogfaa andre Tjenere

<h4>4546</h4>293. Men da vi nu , saalenge vi leve i denne Verden , imidlertid ikke kunne fole eller forståa Andet , end at vi leve , og Gud opholder i Live dem , forn sove og slet Intet gjore eller virke , hvorfor stulde vi da Me tilskrive ham saa stor en <b>Visdom</b> og Magt , at han kunde opholde <b>Sjelen</b> ogsaa i Doden ? Thi medens den endnu boer i Legemet , spottes den paa mange Maader i Sovne , idet der i Dromme foregjogles den mangeslags indbilote Billeder , som itte ere virkelige ; dog lever den alligevel, hvorvel den er saaledes overveldet af Sovnett , at den itte kan handle eller vere virkfom ; men Legemet ligger som en Stok uden nogen Folelse . Derfor kan Ingen lettelig sige , hvad Sovn er , hvad Vaagetthed, eller hvad <b>Sjelen</b> er o . f . v. 294. Hidtil have vi nu talt om de « Helliges og de Ugudeliges Grav , og det er vausteligt at sige noget Sikkert derom . Heller itte er det os saa magtvaaliggende faa noie at vide , hvorledes det forholder sig med de Ugudelige , da vi jo maa tro Christi Ord , der siger om den dode <b>Pige</b> Marc . 5 ^ 39. : Pigen er Me dod , men hun sover . len saadan Erkjendelse eller Tro have de Gudfrygtige ttok . 295. De Theologer , som kaldes Scholaftikere , tale om sire Kredse eller Steder ide Dodes Rige . Dett forste kalde de liuil ) iiB pati-um eller Fedrenes Kreds ; der have de hensat dem , som ere dode sor Christi Fodsel , Lidelse og Opstandelse , som have ventet paa Christi Gjenlosning og , Nedfart til Helvede fra dette Liv . Thi Himlen var endnu Me aaben . Men de forståa selv itte , hvad de sige eller mene . Vi kalde det Abrahams Skjod , saaledes som Evangeliet lerer os , medens de kalde det liinduB Mti-uiu , det Sted , hvori alle Hellige og Gudelige for Christi Komme befinde sig . Og jeg vil itte kives om Navnet , efterfom man med Vishet » ved , at det dengang var Abrahams Skjod , forn nu er Christi Skjod . 296. Thi eftersom Moses og Elias tale med Christo paa Bjerget , Matth . 17 , 3. , fremgacter deraf klarlig , at hatt har alle disfe Hellige i sin Haand og Magt , og at han er deres Gud . Men de sove og hvile , som man ved om alle Gudfrygtige , dog faa , at Christus kan opvcekke dem , nåar det lyster ham . Derfor er det ikke fornodetti , at bede for dent , og endnu langt mindre fornodent er det , uden Nytte at bortkaste faa mange Penge foråt forlose Sjelene fra Skjersilden , faaledes forn det stede i Pavedommet . 297. Have nu Augustinus eller andre Fedre en anden Mening herom , saa skulle vi vide , at man maa lese deres Skrifter med ret Skjott , og at man Me stal gjore nogen Troesctrlikel af Fedrenes Ord eller Meninger , ved hvilken Daarstab der for i Tiden er bleven indfort grueligt Morke i Kirkeu . 298. Gregor * ) er Ophavsmanden og Austifteren af de mange

<h4>4970</h4>199. Dette er en faadan Maade at tale paa , forn er almindelig og fedvaulig for Gud . Moses taler Me saaledes , luristerne og Philosopherne heller ikke ; men det er en Forjettelsens Theologi og Guds Ord , der giver Forjettelsen , talt af ham i Himlen blandt Englene ; men vi forståa den itte , forent » hans Raad fuldbringes . Deraf kommer det , al Herrens Raad faa ofte prises hos Propheterne . Og Ps . 107 , 11. staaer der : de foragtede den Hoiestes Raad . Dette stal man nu lere , paadet vi deraf kunne faa Skjott paa , hvad der er den rette Theologi og Evangeliets Lere . 191. Philosophernes og Lovens Lere er en anden Lere ; Moses har ikke en saa ferdig Tunge , men dett er ham noget tung ; han stammer og taler utydelig ; han kan Me ret udtale dette Ord . Men han , forn giver Forjettelsen , Frelseren , Jehova , han taler saaledes . 192. Men Naturen , som er daarlig , og Kjodet , som er fordervet ved Synden , forstaaer itte dette , forend Lidelsen er - til Ende , forn Herren siger til Moses : du stal se mig bag til , 1 Mos . 33 , 23. , da han bad om , at Gud vilde <b>vise</b> ham sit Ansigt ; det er : forst nåar Gjerningen er fnldbragt , stal du se mine Tanker . Saaledes se ogsaa Joseph og Jakob tilligemed hans Sonner ham her ikke i Ansigtet , men bag til . Thi da Joseph solgtes , og hans Fader forgede derover , da skjultes Gnds Ansigt og ingen Gud lod sig se ; men hele Verden syntes at vere fuld af Djevle . Mett nu , da hans Naad er opfyldt og aabenbaret, fe de lydelig og erkjende nu ogsaa Guds gode , velbehagelige og fuldkomne Villie . 193. Saaledes siger ogsaa Hagar ovenfor ( Kap . 16 , 13. : har jeg ogsaa hidindtil seet ester den , som mig ser ? som om hun vilde sige : da Gud saa mig , saa itte jeg ham ; nu ser jeg hans Nyg , som hidtil har seet mig , medens jeg Intet vidste af hans Naad og Gjerninger. Saaledes siger ogsaa Christus til Petrus Joh . 13 , 7. : hvad jeg gjor , ved du Me nu . Du vil komme mig i Forkjobet og lere mig , hvad jeg stal gjore ; men det gaacr itte , Petrus ; thi det sommer sig for dig at taale min Haand . Lad mig faa handle ; men sidenefter stal du nok erfare og forståa , hvad jeg har ment med denne Fodtuettiing. 194. Saaledes er nu dette den Maade og <b>Vis</b> , hvorvaa Gud styrer de Hellige , og Kirken og Guds Meuigheds rette <b>Visdom</b> , nemlig at man ikke forarges ved Guds Naad og itte tager Anstod af det Ansigt , hvormed hau <b>moder</b> os . Thi undertiden pleier han at tilhylle og skjule det , saaat vi ikke kunne kjende eller se det . Vistnok er han en Livets , Wrens , Frelsens , Gledens og Fredens Gttd ; det er Guds sande Ansigt ; uten han skjuler det undertiden og paatager sig en anden Maste , hvori han <b>moder</b> os som en Vredens , Dodens og Helvedes Gud . Derfor fe til , at du er <b>vis</b> og forstandig , lerer og horer , hvad dog hans Mening og Hensigt er med denne usedvanlige og paafaldende Skikkelse . Thi det er hans Hensigt , at dn stal ydmyges og lere at taale Herrens Haand og det Ansigt , som han aabenbarer dig , og bie derefter.

<h4>5622</h4>var en hellig og priselig Sikkerhed , da han endnu flet ikke havde solt noget Ondt eller nogen Fordervelse . Saaledes ere ogsaa vi sonderknuste og ydmygede i Doden og have dog Livet ligesaa ner , som Adam havde Doden ; thi Forjettelsen Mare . 16 , 16. er sikker og sanddru : hvo som troer og bliver dobt , stal blive salig ; her er Livet ganske ner os , som ere i Doden , ligesom Doden var Adam ner . Men hvo der troer , ligesom Jakob her med sterk Tillid venter sig Livet , han agter Doden for Intet ; thi Troen bringer Livet ner til og slutter faaledes : om jeg end doer , saa er jeg dog dobt og troer paa lesum Christum ; derfor er jeg levende og salig . 156. Ligesom nn Adam kunde rose sig af at leve , og dog alligevel ogsaa stulde tro dette Ord : paa hvilken Dag du eder af det , skal du ds Doden , og frygte for Doden , saaledes skulde vi derimod ikke frygte for Doden , men huste paa Livet . Thi Gud , som til Adam sagde : Doden er dig ner , den samme Gud siger ogsaa til os : Livet , Freden og Saligheden er ner ved Eder . Og Gud , som taler saaledes med os , kan ikke lyve eller bedrage , som Paulus siger Tit . 1 , 2. : Gud , som Me lyver , har lovet os det evige Liv , hvilket er et Ord af den Tro , forn allerede har Livet i sig , hvorvel det er stjult ; thi Troen begynder Livet i os og har Livet i sig selv . 157. Vi som tro have denne Begyndelse , at vi , om vi end nu fole Dodeu , dog ikke frygte for den , ligesom de , der plages af en ond Samvittighed , blegne og forferdes endog uaar de blot hore Doden nevnes . De gudfrygtige og hellige Mennester derimod foragte Doden . Da saaledes St . Agnes blev slebt afsted til Fengsel og Pinsler , sagde hun , at hun var tilmode gauste forn om hun fortes til en Dauds . Kjere , hvorfra fik dog denne unge <b>Pige</b> faadant hoit Mod , at hun flet ikke frygtede eller forferdedes , men var glad og veltilmode , ret forn om man satte sig tilbords med hende og vilde traktere hende kostelig ? Det var ikke en epikureisk Foragt for Doden , men en ret <b>Visdom</b> og et rigtigt Skjon , hvori hun kunde slutte og vere <b>vis</b> paa , at Livet var hende ganste ner . Derfor spottede hun Djevelen og Doden og lo ligesom af dem ; thi Doden var hos hende opslugt af Livet . 158. Dette er vor Theologi , som vi lere , gauste modsat den , som de blinde og taabelige Scholastikere og Papister have holdt saa hart paa , hvilke Intet vide herom og foragte Troen . Lad os derfor hore Patriarken Jakob , der taler om Doden som om en Sovn . Thi sporger du ham : hvad , kjere Jakob , er du ikke bange for Doden ? saa svarer han : vistnok stal jeg do og legges i Graven ; men Gud lever , som har forjettet os det Land , hvortil han vil fore Eder ; mig derimod vil han stytte til et andet Liv , forn er meget bedre ; thi han har lovet det .

<h4>5832</h4>2 , 3. : af Zion stal udgaa en Lov og « Herrens Ord af Jerusalem . Ligesaa Rom . 1 , 16. : Evaugcliet er en Guds Kraft til Salighed for hver dcv , forn troer . I Ordet er guddommelig Kraft og <b>Visdom</b> ; hvo der troer det , han stal blive salig . 164. Saaledes er det et Rige med Kraft i Ordet imod Doden , Synden og Djevelen , mod al deres Magt og Tyranni ; det har Kraft til at hjelpe , frelse og beskytte til den evige Salighed. 165 Om disse Ting vide loderne og Rabbinerne Intet , Papisterne og Tyrkerne heller Intet ; men os sommer det , flittig at indprente Folk dette og vel at merke og fastholde denne vigtige Forskjel mellem Christi Rige og de andre Niger , ogsaa Davids . Thi det mente Jakob : Davids , min Sons , Nige , som ikke vil kunne regjeres uden Sverd og legemlige Vaaben , stal ikke vedvare bestandig , men efter det stal Schilohs Rige folge , som alene stal regjeres med Ordet . Som Christus siger Mare . 16 , 15. : gaacr bort i al Verden og prcediter Evangelium sor al Skabningen . Thi Ordet er kraftigt ; det kan hjelpe og frelfe fra Dodens og Djevelens Haand , dertil fra Helvedes Magt , og kan sette Folk over i Guds Rige . Derfor stulle Folkene adlyde denne Konge , det er , de stulle regjeres med Ordet ; Ordets Forkyndelse vil det dreie sig om ; det stal vere det Merke , hvorpaa Christi Nige skjelnes fra denne Verdens andre Riger , der regjeres med Sverdet og udvortes Magt . Vore Stridsvaabeu , siger Paulus 2 Cor . 10 , 4. , ere itte tl > detige , men mirgtige for Gud til at forstyrre Befestninger o . s . v. Jeg absolverer dig , jeg dober dig , jeg erklerer dig for et Christi Riges Barn ; jeg forkynder dig Syndernes Forladelse , jeg forjetter og tilbyder dig Seier imod Djevelen . Tro , saa stal du blive salig . 166. Saaledes taler Jakob saare tydelig som den , der er oplyst af den Hellig-Aand ; han saa starpt langt borte , da han sagde : og Follene stulle hlrnge ved ham . Det samme Ord ( littehat ) brnges ogsaa Salom . Ordspr . 30 , 17. : et Die som spotter siv Fader og foragter at hcenge ved sin <b>Moder</b> , det stulle Navne udhugge ved Bcrttcn . Dette Sted er feilagtig oversat i den almindelige latinske Oversettelse ( thi den er ikke af Hieronymus ) . Men Salomo taler her om det fjerde Bud og den Wre , forn Bornene ere styldige at <b>vise</b> sine Foreldre . Og det tjener saare vel til at bekrefte Meningen af denne Text , at litlehllt betyder Lydighed . Thi Evangeliet er Noget , forn hores , forn der staaer Es . 53 , 1. : hvo troede det , vi havde hsrt ? Denne Schiloh stal ikke herske ved Sverd , ved Ild eller anden udvortes Magt , men ved Troens Forkyndelse og <b>Lerdom</b> , og ham stulle ikke alene loderne , men alle Jordens Folk adlyde . 167. Det hebraiste Ord Am er velbekjendt , og betyder et Folk . Pnn samme Mnnde som her siger Esnins Kap . 31 , 7. 8. 9. , at alle Afguder i hele Verden stulle opheves og bortkastes , og at der i Alverden stal blive Enighed i Lydighed mod den eneste Gud og hans Dyrkelse . Thi saa siger Propheten der : paa den samme Dag skulle de fortaste hver sine Solvasgndcr og sine Guldafgndcr , som Eders Hender gjorde , Eder til Synd . Og Assur stal sålde , itte ved Mauds Svlrrd , og et Svlrrd , som ilte er et Menneskes , stal sortlere ham , og han stal fly for Svlrrdet , og hans unge Karle stulle vere statstyldige . Og hau stal gaa sin Klippe sorbi af Frygt , og hans Fyrster stulle forstrettes

<h4>6099</h4>437. Det er os nok , at Skriften vidner , at fra Verdens Begyndelse af ere de itte gauede fortabt , forn have troet paa <b>Kvindens</b> Sed , og at de ikke ere blevne glemte , men samlede til sit Folk . Men hvad dette er for et Sted , kan Ingen sige . 438. Ovenfor anforte jeg en fra det Legemlige hentet Lignelse om Fosteret i Moderlivet . Thi der er intet levende Menneste , som selv kan vide , hvor han var de forste to Aar , da han dels levede i Moders Liv eller , efter Fodfelen , erneredes af sin Moders Melk . Han ved Intet om , hvorledes Dagene og Netterne eller Tiderne vare , eller hvem de vare , som passede og pleiede ham ; og dog levede han dengang , og <b>Sjel</b> og Legeme vare forenede med hinanden og han var stikket til alle naturlige Gjerninger . Saaledes er dette et sikkert Bevis for , at Gud vil opholde Mennesket paa en underfuld Maade , forn er Mennesket selv ganste übekjeudt . Betragt de stakkets Meuuester , som lide af den faldende Syge eller Vanvid , hvori de ere faa rent fra sig selv , at de ganske synes berovede sine Sandsers Brug . Ja endog om Natten , nåar vi ligge og sove , vide vi itte , paa hvilket Sted vi ligge , om vi ere hos Venner eller Fiender . 439. Hvad er dog det for en stor Daarstab og Formastelighed , at vi saaledes ville fatte den guddommelige <b>Visdom</b> med den trange mennestelige Fornuft ! Kunne vi itte tenke os , at han vil bevare og opholde <b>Sjelen</b> , uden at jeg just ved , hvor jeg er , uagtet jeg dog er paa et bestemt Sted , da han jo dog daglig gjor det , som de nevnte Exempler <b>vise</b> .

<h4>6103</h4>440. Derfor stulle vi give Gud Mren og ere ham ved at anfe hans <b>Visdom</b> for uendelig og tro om ham , at han har flere og underfuldere Maader at opholde os paa , end vi kunne begribe med vore Sandser og vor Fornuft . Han bevarer og opholder os paa een Maade i Moderlivet , paa en anden i Vuggen ; paa een Maade i Sovne og atter paa en anden , nåar vi ere fyge . Vi have nok iat vide , at de Hellige i det gamle Testamente dode i Troen paa den Christus , som stulde komme , og at de Gudfrygtige i det nye Testamente , som nu do i Troen paa den samme Christus , der nu er kommen , ere samlede til sit Folk . 441. Men hvilket eller hvordan : dette Sted er , vide vi itte . Men det er vist , at det itte er noget ondt Sted , og de flet itte have nogen Pine eller Plage , men hvile i Guds Naade . Ligesom de i dette Liv sove sodt og ere under Guds og de kjere Engles Bestjermelse og itte frygte for nogen Fare , vare de end midt iblandt Djevle , faaledes ere ogsaa efter dette Liv de Gudfrygtiges Sjele i Guds Haand , sont der staaer <b>Visdom</b> . Kap . 3. 442. Men hvorledes denne Hvile er beskaffen , det har Ingen nogensinde vidst nndtagen de , som opstode med Christo , eller som han opuakte fra de Dode , saafom Adam , Eva , Noah , Abraham og Andre , ligesaa Enkens Son i Nain og den Overstes , lairus ' s Datter . 443. Meu derom er der iugen Tvivl : de , som samles til sit Folk , hvile i Fred . Ja , siger du ; men hvorledes gaacr det til ? Sig du mig , hvorledes det gaacr til , at dn sover , eller paa hvad Maade Fosteret opholdes i Moderlivet . Din <b>Moder</b> folte vel , at du rorte dig og levede i hendes Liv ; men du ved ikle , hvorledes du levede , hvorledes du kunde tåge med dine Hender , fe o . f . v. Du har ogsaa diet div Moders Bryst uden at vide Noget derom ; og dog vil du nu vide og forståa , hvorledes Gud bevarer og opholder dig i Fred efter din Dod ? 444. Derfor stulle vi tro , at der er et serligt Sted for de Udvalgte, hvor de Alle hvile med Undtagelse af dem , forn Gud vilde stulde opstaa med Christus . Thi det behagede Gud , med deres Opstandelse at ville ere de Forstesodtes Dag , Christi Opstandelsesdag . Jeg troer , at ogsaa Joseph dengang blev opvakt ; de laa Allesammen i Gravene indtil Christi Komme . Det stulle vi tro , og ikke saa nysgjerrig eller noie forste efter denne Hviles Bestaffenhed . Leg meget mere Vind paa at blive En af Guds Folk og tro , at Christus er avstanden fra de Dode . Gud stal nok forge for , at du er med blandt de Udvalgtes Folk , han forn har opholdt dig snnd og levende i Moderlivet , i Vuggen og i Sovnen . 445. Det er sandelig en underfuld Ting , at Gud i at sove , spise , vaague og flere andre Ting gjor os lige de ufornuftige Dyr . Thi Mennestets <b>Sjel</b> sover saaledes , at alle Sandser ere ligesom begravede , og vor Seng er ligesom en Grav , som dog intet Felt eller Ondt har ved sig . Saaledes er der heller itte nogen Pine paa dette Sted , hvorhen de Dode kvmme , men , som man pleier at sige , de hvile der i Fred , ligesom man heller itte ser noget Spor til Smerte eller Moie hos dem , som vaagne op af Sovne , hvorvel de itte vide , hvad der imidlertid er foregaaet , medens de fov . Det Samme ster ogfaa , nåar et Menneste henrykkes eller kommer i Ekstase , saaledes som Augustin forteller om en Henrykkelse , som han selv og hans <b>Moder</b> havde , saaat de Begge , da de kom til sig selv igjen , itte vidste , paa hvilket Sted de befandt sig .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008082512001'target=_>Luther, Martin, 1863, Dr. Martin Luthers Udførlige Udlæggelse af første Mosebog</a><h4>272</h4>208. Foråt ilke Nogen skulde tro , at <b>Kvinden</b> var udelukket fra det tilkommende Livs Herlighed , saa omtaler Moses her Begge , Manden og <b>Kvinden</b> ; thi <b>Kvinden</b> synes jo at vcere noget anderledes end Manden , ligesom hun da ogsaa har andre Lemmer og en svagere Tanke og Forstand. Og hvorvel Eva var en herlig Skabning og Adam lig , hvad angaaer Guds Billede , det er i Retfcerdighed , <b>Visdom</b> og Salighed , saa var hun dog en <b>Kvinde</b> . Thi ligesom Solen er en herligere og cedlere Skabning end Maanen ( hvorvel ogsaa Maanen er saare Mn og herlig ) , saa var heller Me <b>Kvinden</b> Manden lig i 3 - Ere og Vcerdighed , omendskjMt hun var en saare skM Guds Gjerning . 299. Men derfor stiller Moses her Begge sammen , Mand og <b>Kvinde</b> , og siger : Gud stabte dem Mand og Kvinoe , fordi han vil <b>vise</b> , at ogsaa Eva er skabt af Gud og delagtig i Guds Billede og Lignelse , ogsaa i Herredømmet over Alting . Saaledes er endnu den Dag idag <b>Kvinden</b> delagtig i det tilkommende Liv , som St . Petrus siger 1 Petr . 3 , 7. , at <b>Kvinderne</b> ere Medarvinger til Livets Naadegave . Og i Husholdningen er <b>Kvinden</b> delagtig i Regimentet og sidder i lige Besiddelse af Nornene og Godset ; og dog er der en stor Forstjel mellem KjMnene . Manden er ligesom Solen paa Himlen , <b>Kvinden</b> ligesom Maanen ; derimod ere Dyrene ligesom Stjernerne , over hvem Solen og Maanen herske . Derfor skulle vi af dette Sprog forst mcerke , at det er skrevet , foråt <b>Kvinden</b> ikle stal udelukkes fra nogen den menneskelige Naturs Arv , om hun end er en svagere Del end Manden . Om 3 - Egtestanden skulle vi siden tale . 219. For det Andet er her et Bevis mod Hilarius og Andre , som have ment , at Gud stabte Alt paa eengcmg . Thi her saa vi Grund og Bevis for vor Mening , at disse Skabelsens sex Dage ligefrem vare sex naturlige Dage , fordi Texten her siger , at Adam og Eva bleve stabte paa den sjette Dag . Denne Text kan man ikke fordreie . Men i fslgende Kapitel siges der , i hvilken Orden Mennesket stabtes , nemlig at Eva stabtes en Stund efter Adam , ikke af en lordklump faaledes som Adam ,

<h4>333</h4>28. Her kommer Moses atter til den sjette Dags Gjerning , og viser , hvoraf Mennesket , som skulde eie og dyrke Jorden , er kommet , nemlig at Gud havde gjort det af Stov af Jorden , ligesom en Pottemager med Haanden dcmner et Fad af Ler . Derfor sagde han ikke ovenfor , ligesom om de andre Skabninger : Jorden udgive et Menneske ; men han sagde : lader os gjne et Menneske , for derved at <b>vise</b> Menneskets soerdeles Herlighed og aabenbare Guds sceregne Naad , ester hvilket han handlede ved Menneskets Skabelse . Hvorvel Mennesket sidenefter voxer og tåger til ganske paa samme Maade som de andre Dyr ; thi Livsfrugten dannes i Moderlivet ganske paa samme Maade , har sin Begyndelse , sin Tid , lever , voxer , tåger til o . s . v. , saaledes at der i dette Tilfcelde ingen Ulighed er mellem et drcegtigt Kvceg og en svanger <b>Kvinde</b> . Men i den ftrste Skabelse viser Moses en stor Ulighed . Thi den menneskelige Natur er stabt efter et scereget velbetcenkt Raad og i Guds <b>Visdom</b> , og tilberedet og dannet med Guds Finger . Og denne Ulighed , som sinder Sted mellem Mennestets og Kvcegets Skabelse og Oprindelse , viser ogsaa Sjlrlens Utwdelighed , hvorom jeg kort ovenfor har talt . Thi hvorvel alle andre Guds Gjerninger ere saare at forundre sig over og meget herlige , saa viser dog det , at Mennesket er den allerherligste Skabning , at Gud holder Raad med sig selv og gaaer frem paa en anden Maade , da han staber det ; han lader ikte Jorden danne og frembringe det af sig , ligesom Dyrene og Trceerne , men han danner det selv efter sit Villede , saasom det skulde vcere delagtigt i Gud og nyde hans Hvile . Derfor var Adam , forend han dannedes af Herren , en dod lordklump , som laa der ; denne tåger Gud og gjor deraf den allerstjonneste Skabning , som er delagtig i Udodeligheden . 29. Hvis Aristoteles hMte dette , da vilde han le meget deraf og holde det for den taabeligste Fabel , om det end er ganske listigt at hore , at Mennesket efter sin forste Oprindelse var en lordklump , men ved Guds <b>Visdom</b> blev saaledes dannet og stabt , at det kunde vcere udodeligt. Thi hvorvel uogle af Philosovherne , saasom Sokrates og Andre , have ment og lcert , at Sjcelen er ndodelig , saa ere de dog af de Andre blevne udleede , ja ncesten forstodte og fordomte . Men er ikke det en stor Daarlighed af Fornuften , at den tåger faadan Forargelse deraf , da den dog endnu ser , at Mennestets Avling er en Gjerning , som er saare at

<h4>400</h4>Modrene havde uden Smerter baaret og fodt Wrn , hvilke ogsaa vare blevne opdragede uden sacidan Jammer og stor Mole . 9 ft . Men hvo kan dog med Ord beskrive denne Ustyldighedens store Herlighed , som vi have tabt ? Vistnok har Naturen beholdt Mandens Kjcerlighed og Attråa til <b>Kvinden</b> , ligesaa Frugterne as Bornecwlingen , men med gruelig Besmittelse as Utugten og med stor Kummer og Smerte ved Fodselen . Dertil kommer Skam , Blusel og Forvirring , ogsaa blandt 3. Egtefolk , saaofte de ville nyde den tilladte Omgang . Saaledes staaer overalt Arvefyndens store og jammerlige Skade os målet for Mnenel Vistnok er det ikke ondt at avle Born , Velsignelsen derved er ogsaa god ; men disse Ting ere saaledes odelagte og fordcervede ved Synden , at Me 3 - Egtefolk kunne benytte dem nden Skam . Men dersom Adam var forbleven i Ustyldigheden , da havde Intet deraf fnndet Sted ; men ligesom Mand og <b>Hustru</b> uden Skam sidde tilbords og spise med hinanden , saaledes havde det ogsaa vceret en synderlig 2 - Ere at avle og sode Bprn , og ingen Skam eller Blusel . Men jeg vender tilbage til Moses . 91. Her anordnes , som sagt , Kirken , forend endnu Husholdningen var til . Thi Herren prcediker her for Adam , og forelcegger ham et Ord , som , om det end er kort , dog er vcerdt , at vi tale noget udforligere derom. Thi denne Prcediken vilde have vceret Adams og vores Alles Bibel , dersom han var bleven staaende i Ustyldigheden ; heller Me havde vi da tiltrcengt Papir , Blcek , Penne og saa utallig mange Boger , som vi nu maa have , og kunne dog ikke opnaa Tusindedelen as den <b>Visdom</b> , som Adam havde i Paradiset . Men denne korte Prcediken vilde have sammenfattet og indesluttet hele <b>Visdommens</b> Studium og havde vist os afmalet ligesom paa en Tavle Guds Godhed , som havde stabt Naturen ren og fuldkommen , uden nogen Skade og Brcek , som sidenefter fulgte vaa Synden . 92. Og denne Prcediken , hvilken , som Texten viser , Adam alene Horte , stede vaa den sjette Dag , og Adam meddelte den siden til Eva . Og hvis de Begge ikke havde fynder , saa . , havde Adam foreholdt sine Efterkommere dette eneste Bud , hvoraf der da var bleven de bedste Theologer, de lcerdeste lurister og de mest erfarne Lceger . Nu har man Boger, som ikke ere at tcelle , hvori man underviser Theologer , lurister og Lceger ; og dog kan alt det , som vi lcere ved Bogernes Hjcelp , neppe kaldes en Levning imod hin <b>Visdom</b> , som Abam havde ost as dette ene Ord . Saa aldeles er Alt fordcervet og tabt ved Arvesynden . 93. Derfor vilde dette Kundstabens Trce paa Godt og Ondt , eller dette Sted , hvorpaa saadcmne Trceer vare plantede i stor Atcengde , have vceret den Kirke , hvor Adam om Sabbathen havde forfamlet sig med sin kommende Slcegt og hvor de , efter Forfrifielse af Livets Trce , havde lovet Gud og priser ham , fordi han havde overgivct dem Herredommet over alle Skabninger paa Jorden , faaledes fom den 148. og 149. Psalme ncesten indeholde og <b>vise</b> en saadan Taksigelsens Form , idet de byde Solen, Maanen , Stjernerne , Fiskene o . s . v. at prise Gud ; hvorvel ingen Psalme er saa hoi og herlig , at jo Enhver as os havde kunner gjore en bedre og fuldkomnere , hvis vi vare avlede af Adam i Ustyldighedsstanden. Og Adam vilde have priset Guds storste Velgjeruing , at han med sin hele Slcegt var stabt i Guds Villede , havde formaner sine Efterkommere til at leve hellig og uden Synd , dyrke Haven trolig og bevogte den omhyggelig , ogsaa med hoieste Flid vogte sig for Kundstabens Trce paa Godt og Ondt . Og der havde Mennesket saaledes havi dette ydre

<h4>444</h4>136. Hvor besad dog Adam en herlig , rig Erkjendelse og <b>Visdom</b> ! Men hvorvel han ogsaa tabte saare Meget deraf ved Faldet , saa er det dog min Tro , at Alt , som endnu staaer i alle <b>vise</b> Folks Boger , der ere strevne i saa mange hundrede Aar , fra den Tid af , da disse Videnstaber forst opfandtes , alligevel tilsammentaget ikke kunde komme op mod den <b>Visdom</b> , som ogsaa efter Synden forblev hos Adam , men som derpaa efterhaanden forduukledes hos hans Efterkommere , og nu ncesten ganske er forsvunden . 137. Fremdeles maa vi her atter mcerke , at Moses endnu holder paa at beskrive den sjette Dags Gjerning . Thi hvad han kortelig sagde i Kap . 1. V . 26. : Lad os gjsre et Menneske , det har han i dette Kapitel villet udfore rigeligere og vidtloftigere , og med mange Beviser og Grunde adstille Mennesket fra Dyrene . Derfor tilbringer han hele Kapitlet med at forklare , hvorledes Mennesket blev skabt . Om Manden sagde han , at han blev skabt af Jord , og at Gud indblceste Livets Aande i hans Ncese , at han siden ogsaa bragte for ham hele Samtingen af alle Dyr . Men da nu Adam ikke kunde sinde sig nogen Medhjcelp blandt disse , saa gjorde Gud <b>Kvinden</b> til en Medhjcelp og Deltager ved Borneavlingen og den menneskelige Slcegts Opholdelse . Thi Gud vilde ikke , at Adams Efterkommere skulde stabes af Jord paa samme Maade som han ; men han vilde , at Mennesket skulde forplante sig , ligesom Dyrene . Thi hvad angaaer det naturlige Liv , saa svise vi , drikke , avle og avles ligesom Dyrene. Og Moses lader sig det vcere magtpaaliggende at adstille og afsondre Mennesket fra Dyrene , for derved at antyde , at Mennesket efter sit naturlige Liv skulde vcere delagtigt i det aandelige og evige ; men alt dette henhorer , som sagt , til den sjette Dags Gjerning . Thi efterdi Gud havde sagt : vorder frugtbare og mangsoldige , saa var det fornodent at forklare , hvorledes <b>Kvinden</b> blev stabt , forenet og gift med Adam . 138. Men dette tjener til at gjore os visse og sikkre i den Me-

<h4>452</h4>Fcedre og Forfcedre ; men for deres Skyld vige vi dog ilke af fm Skriftens Anseelse . 141. Og Aristoteles har rigtig og smukt sagt , at det er meget bedre at stjcenke Sandheden sit Bifald end at hcenge altfor fast ved dem , som ere os kjcere og vore Venner , og at det iscer sommer sig for en Philofoph ut gjore dette ; thi hvis baade Sandheden og Vennen ere os kjcere , saa stal man dog foretrcekke Sandheden for Vennen og agte den mere . Naar nn en Hedning byder at gjore Saadant i verdslige Sager , hvor meget mere er det da at gjore i de Sager , som have aabenbare Vidnesbyrd af Skriften , at vi ikte stulle foretrcekke Mennesters Anseelse fremfor den hellige Skrifts ? Thi Mennester kunne feile , men Guds Ord er Guds <b>Visdom</b> selv , og den allersikkreste Sandhed . 142. Men hvad nu angaaer denne Historie , saa sig mig , hvad kunde dog lyde og siges fabelagtigere , nåar man vilde folge Fornuften? Vilde der vel vcere Nogen , som vilde tro denne Historie om , hvorledes Eva blev stabt , hvis det ikke saa klart og udtrykkelig sagdes i Skriften ? Thi her ser du ved alle andre levende Vcesener det Modsatte . Thi hvad der fodes levende , det avles af en Mand og en <b>Kvinde</b> , og fodes til Verden af <b>Kvinden</b> ; men her stades <b>Kvinden</b> selv af Manden , hvilket er en ligesaa forunderlig Gjerning , som at Adam af en lordklump dannes til et levende Menneske . Hvis du vilde se bort fra Skriften og folge Fornuftens Dom , da maatte Saadant vcere de allerstorste og groveste Logne og Fabler . Ligesom Aristoteles siger , at man hverken kan cmtage noget forste eller noget sidste Menneste ; hvilket ogsaa Fornuften vilde tvinge os til at sige , dersom ikke denne Text var . Thi hvis du fastscetter , at dette er sandt ( og saaledes forholder det sig jo i den hele Skabning ) , at intet Levende kan fodes uden af en Han og en Hun , saa kan man jo heller ikke scette noget Menneste som det forste . 143. Dette skulde man vel ogsaa sige om Verden , om hvilken derfor Philosopherne have stuttet , at den er evig , hvilken Mening Fornuften med al Magi vil folge , om man endog har Grunde og Beviser , hvormed det godtgjores , at den ikke er evig . Thi hvad vil Fornuften sinde for en Vegyndelse i det , som Intet er ? Hvis du nu altfaa siger , at Verden har en Vegyndelse , og at der var en Tid , da der endnu ikke var nogen Verden , saa folger deraf ligefrem , at der for Verden var Intet . Der folger ogsaa uendelige andre Urimeligheder , som volde Philosopherne meget Bryderi og have forcmlediget dem til at mene , at Verden er evig og hverken har Vegyndelse eller Ende . Men hvis du vilde sige , at Verden er uendelig , saa ovstaaer strår , ogsaa et andet Uendeligt , nemlig at Mennester altid maatte komme efter og folge paa hinanden . Men nu kan Philosophien ikke tillade mange uendelige Vcesener ( iMnita ) , og her maa den dog tillade det , fordi den ikke vil vide af nogen Begyndelje vaa Verden og Mennestene . Saadant selvmodsigende Morke har givet Epiknrceerne * ) Anledning til at sige , at , Verden og Mennestene beroede vaa et Tilfcelde og ikke havde nogen <b>vis</b> Aarsag , at de ogsaa vaa Slump og tilfceldigvis atter vilde gaa til Grunde , ligesom Dyrene falde hen og do , som om de aldrig havde vceret til . Deraf folge da igjen andre Ting , nemlig at der enten ingen Gud er , eller at Gud ialfald stet ikke bryder sig Noget om Verden og Mennestene . Se , vaa saadanne vildfarende

<h4>457</h4>144. Derfor er et nvttigt , at man ser , hvorledes vor Fornuft eller <b>Visdom</b> ilke kan komme langt nok , nåar den vil maale og betragte Skabningen. Thi hvad kan en Philosoph vide om Himlen eller Verden , da han jo Ne ved , hvorfra den kommer , eller hvad der tilsidft bliver af den ? Paa os felv fole og se vi , at vi ereMennester ; men at vi have den eller den Fader og <b>Moder</b> , kunne vi ingenlunde vide ; det maa vi tro . Lcengere gaaer Ne hele vor Erkjendelse og <b>Visdom</b> , end saalangt som Natsria og gaa , det er , saalangt som den kan tcenke og se ; hvorvel vi underliden skammelig feile og fare vild ogsaa heri . Men hvad der er oauß3 . 6 Moi6QB og tinali » , det er , hvad det er som fra Forst af bevirker og foraarsager en Ting , og hvorledes eller hvor den stal ende , og hvortil den stal tjene , det kunne vi paa ingen Maade <b>vise</b> , iscer ( hvad der dog rigtig er ynkeligt ) , nåar vi tale om den Verden , hvori vi dog ere og leve , ligesaa nåar vi ville tale eller tcenke om os selv . Men er det Me rigtig en elendig og ussel <b>Visdom</b> ? 145. Aristoteles siger : Solen og et Menneske avle det nceste Menneske. Dette er vel nok ; men folg du denne <b>Visdom</b> , saa vil dn tilslut komme did , at du maa slutte , at Mennesket og Solen ere evige og uendelige. Thi du vil aldrig finde et Menneske , som enten er Begyndelsen eller Enden , ligesom jeg ikke kan finde min egen Persons Begyndelse og Ende , hvis jeg vil vide det med fuld Vished , og ikke blot tro det . Men hvad er nu det for en <b>Visdom</b> eller Kundstab , nåar man ikke forstaaer Ende og Begyndelse eller causa Knali » og 6 tNoi6NB ? Thi at vi kjende en Tings Form og Skikkelse , det er ikke stort Andet , end at en Ko kjender sin Baas , og , som man siger i Ordsproget , ser paa en målet Vceg . Derfor kan man ogsaa Heras se , hvad Arvesynden er for et strcekkeligt Fald , ved hvilken vi have tabt denne Erkjendelse , saa at vi hverken kunne vide vor egen Begyndelse eller Ende . 146. Naar nu Pluto , Cicero og andre Philosopher , som ere de bedste , tale om , at Mennesket gaaer opreist og bcerer sit Hoved loftet , medens Dyrene boie det ned mod Jorden ; fremdeles nåar de rose Menneskets Kraft , hvorved han kan forståa , skjelne og bedomme ; nåar de endelig ogsaa stutte , at Mennesket er en sceregen Skabning , stabt til Udodellghed : Kjcere , er det da ikke en ringe og ganske intetsigende Ting ? Thi de have det Alt fra , at man saaledes ser Menneskets Skikkelse for sine Vine , ved og kjender den . Men nåar du vil betcenke Menneskets Stof , vil da ikke Fornuften tvinge dig til at stutte , at denne Natur atter maa oploses , og Me kan vcere udodelig ? 147. Derfor stulle vi lcere , at den rette <b>Visdom</b> er i den hellige « skrift og Guds Ord . Thi dette lcerer ikke alene om den hele Skabnings Stof og Form , men viser ogsaa oauß3 . 600 i6Q8 og tiiia , liß , Begyndelsen og Enden paa alle Ting , hvo der har stabt dem , og hvortil de ere stabte . Og nåar vi ikke kjende dette , saa er vor <b>Visdom</b> ikke meget bedre end de ufornuftige Dyrs , hvilke ogsaa kunne se og hore ; men om Begyndelsen og Enden vide de Intet . 148. Derfor er dette en herlig Text , som desto omhyggeligere stal mcerkes og desto fastere troes , fordi den synes ganske og aldeles at vcere imod al Erfaring og Fornuft . Thi her belceres vi om Menneskets Begyndelse, uemlig at ikte det forste Menneske paa naturlig Maade er bleven avlet , ligesom de andre avles af det , saaledes som dog Aristoteles

<h4>480</h4>han foruden sin Villie til at gjore Ondt ogsaa havde Gvnen dertil og kunde rase , som han lystede ? Thi hvis han nu hidsede de vilde Dyr paa os , saa maatte vi Alle do i en Time , i et Meblik . Hvoruel derfor dette Herredomme ncesten ganste og aldeles er tabt , saa er det dog en stor Guds Gave og Velgjerning , at vi endnu have noget Lid et d eraf , ligesom vi ogsaa have Noget af Evnen til at avle Born , hvorvel <b>Kvinderne</b> i Ustyldighedsstanden ikke alene havde baaret og fodt Born uden nogen Ve og Smerte , men ogsaa havde vceret meget frugtbarere . Nu er der tusind Sygdomme , ved hvilke Fodselen forhindres , og det hcender ofte , at enten Livsfrugten ikke lever , eller at 2 Sgtestabet gaaer hen uden Born ; hvilket Altsammen er en Skrobelighsd , der er Straf for Adams strcekkelige Fald og Arvesynden . Saaledes er <b>Kvinden</b> endnu et Hus for Manden , hvortil Manden holder sig , hos hvem han boer , og med hvem han offrer Flid og Arbeide paa at ovdrage og erncere Bornene , som sidenefter siges : Randen stal forlade Fader og <b>Moder</b> og blive fast hos sin <b>Hustru</b> . 173. Desuden bliver det huslige Samliv , foruden anden Ulykke og Jammer , hvoraf der i 3 - Egtestanden for Syndens Skyld er utallig Mget , ogsaa fordcervet og odelagt af forvendte Folk . Ligesom man ikk alene sinder Saadanne , som anse det for <b>Visdom</b> at besvotte det tvindelige Kjon og foragte Mgtestabet , men ogsaa Saadanne , som forlade den <b>Hustru</b> , de have tåget , og kaste al Omsorg for Nornene fra sig . Alle disse nedrive ved sin Ondstab og forvendte <b>Vis</b> denne Gnds Bygning , og er i Sandhed afstyelige Uhyrer i Naturen . Derfor stulle vi adlyde Guds Bud og erkjende vore Hustruer for Guds Bygninger , ved hvem ikke alene Huset ovbygges ved Borneopdragelse og anden i Hnfet nodvendig Tjeneste og Hjcelp ; men ogsaa Mcendene ovbygges ved dem ; thi for dem ere Hustruerne ligesom et Rede og en Bolig , til hvilken de holde sig og hvor de med Lyst og Glcede bo . 174. At Moses tilfoier : og ledte hende til ham , er en stjon Bestrivelse af Trolovelsen eller Brylluvs-Hoitiden , som med Flid maa mcerkes . Thi Adam farer ikke frem efter eget Tykke og griber fat paa Eva , fom nu var bleven stabt , men venter paa Gud , som leder hende til ham , ligesom Christus ogsaa siger Matth . 19 , 6 : hvad Gno har tilsllmmenfoiet, stal Mennesket Me aostille . Thi at Mand og <b>Kvinde</b> paa ordentlig <b>Vis</b> komme sammen , er Guds Ordning og Indstiftelse , derfor beholder Moses sin synderlige og egne Maade at tale paa . Han ledte hende til ham , siger han . Hvem da ? lehovah Elohim , det er det hele guddommelige Boesen , Fader , Son og Hellig-Aand . Disse sige til Adam : Se her , dette er din Brud , med hvem du stal leve og med hvem du stal avle Livsfrugt . Og uden Tvivl tog Adam mod hende med hoieste Lyst og Glcede , ligesom jo endnu i denne fordcervede Natur Brudens og Brudgommens Kjcerlighed til hinanden er hjertelig og stor . Men det var en tugtig , venlig og lystelig Kjcerlighed , uden denne Vellyst , der nu er i Wgtestcmden ligesom en faldende Syge ; saa var ogsaa Trolovelsen saare cerbar og hellig . Men nu kryber og suiger Synden sig ind i Vinene , Brens og alle Sandser . 175. Derfor stal man med Flid lcegge Mcerke til denne Tert , ikke alene imod den strcekkelige Misbrug med Begjcerligheden og den scmdselige Lyst , men ogsaa fordi man med den kan bekrcefte og forsvare den cegtestabelige Stand imod den ugudelige Bespottelse og Forhacmelse , hvormed Pavedommet har krcenket og stjcendet den . Thi er ikke det

<h4>487</h4>Manden forlade Fader og <b>Moder</b> o . s . v. cmfsres af Christo Matth . 19 , 5. saaledes , som om den var talt af Gud.selv og Me af Adam . Men det er ikte vansteligt at forståa . Thi efterdi Adam er ren og hellig , faa kaldes med Rette hans Ord Guds Ord ; thi Gud talte ved ham , og i Wyldighedsstanden vare alle Adams Ord og Gjerninger i Sandhed Guds Ord og Gjerninger . Saaledes forer ogsaa Gud seiv Eva til ham . Og ligesom det derfor er Guds Villie at indscette 3-Egtestanden, saaledes er ogsaa Adam med al Glcede og Hellighed beredt til at tåge Eva til sig , ligesom endnu en Brudgom har hjertelig Lyst og Kjcerlighed til sin Brud , men dog er den besmittet og plettet ved Kjodets spedalske Lyst , der Me var hos Adam , saalcenge han endnu var retfcerdig . 189. Men det er hoilig at undre sig over , at Adam , saasnart som han ser vaa Eva , denne Bygning , der var tåget af ham , forstaaer hende og siger : dette er Ben af mine Ven og Kjsd af mit KM Dette er Ord , Ne af et taabeligt og fyndigt Menneste , som Me forstaaer Guds Gjerning og Skabning , men af et retfcerdigt og vist , som dertil var ovfyldt af den Hellig-Aand , der aabenbarer saadan <b>Visdom</b> , , som endnu er übekjendt for Verden , nemlig at den virkende Aarsag til <b>Kvinden</b> og Wgtestanden er Gud , men at Endemaalet dermed er dette , at Manden i sin <b>Hustru</b> stal have en Bygning og Bolig i Verden . En saadan <b>Visdom</b> og Erkjendelse faaer man Me ligetil af sine fem Sandser og Fornuften , men den er en Aabenbaring af den Hellig- <b>Aand</b> . 181. Det Ord Hllppaam , nu eller denne Gang eller dog , staaer her Me noen Grund , som det vel kunde synes , men udtrykker stjont Mandens Følelse , som lcenges efter dette elskelige Selftab og Samliv med <b>Kvinden</b> , hvilket Ne alene er fuldt af Kjcerlighed , men ogsaa af Hellighed , og ser sig om derefter . Som om han vilde sige : leg har feet vaa alle Dyr , og betragtet de Hunner , som Gud har givet ethvert Dyr til Formerelse og Ovholdelse af dets Art . Men alle disse komme mig ikle ved ; men dette er nu engang Kjod af mit KM og Ben af mine Ben ; med hende begjcerer jeg at leve og adlyde Guds Villie i min Slcegts Formerelse og Ovbyggelse . En saadan fuld Følelse af Kjcerlighed angiver det Ord nuuo eller tauclsiu . , denne Gang . Men vi have tabt denne rene Folelse og Uskyld . Vistnok har Brudgommen endnu en Glcede og hjertelig Lcengsel efter sin Brud , men saare uren og ' strobelig , for Syndens Skyld . Men Adam havde de reneste , helligste og Gud velbehageligste Folelser , hvorved han foraarsages og drives til at sige : det er endelig engang Ben af mine Ben ; Ne af Trce eller Steen , heller Me af en lordklump , men hnn horer mig ncermere til ; thi hun er dannet af mit Kjod og mine Ven . Og fremdeles siger Han . -

<h4>688</h4>Re uden Ink og Medlidenhed se sin <b>Hustru</b> lide saadan Smerte ) , saa er det kommen saa vidt , at ugudelige Folk ofte hellere leve i Horeri end blive 198. Men imod denne de Ugudeliges Mening skulle fromme Christne vide at troste sig og ( hvad der er den rette <b>Visdom</b> ) imod denne Ulykke og Modgang ogsaa holde det Gavn , som er sikkert i Wgtestanden og meget storre end hin . Thi ligesom Pindar i sit Digt til Hiero revser dette hos taabelige Mennester , at de , medens dog Gud pleier at uddele det Gode paa den Maade , at han altid ogsaa lader noget Ondt folge med , ikle forståa , hvorledes de stulle lindre og formilde det Onde med det Gode , som de ogsaa have , hvorimod de Fromme og <b>Vise</b> pryde sit Gode og dermed bedcekke det Onde , som er dem vanlagt , idet de ligesom stille det Gode paa en saadan Plads , at det kan sees af Enhver : saaledes skulde man ogsaa gjore her . Lad det vcere sandt , at den Straf , fom <b>Kvinderne</b> for Syndens Skyld ere underlagte , er stor ; men er dog ikke det langt storre og overgaaer langt al Straf og Plage , at de midt i sin Nod og Jammer have et vist Haab om et udodeligt og evigt Liv ? 199. For det Anoet er heller ikke Korset , Jammeren og Noden uden Frugt ; thi det tjener til , at Naturen derved ydmyges og fornedres ; medens den ellers , uden Kors , ikke kunde tcemmes . 209. For det Treoie forbliver ogsaa under denne timelige og legemlige Ulykke den herlige Modercere og Moderlivets Velsignelse , hvorover ogsaa de Hedninger , som ere noget forstandigere og visere , have forundret sig , og som de have prist som herlig . Saa forbliver derhos ogfaa andre Goder , nemlig at vi alle avles , nceres og bceres i Moderlivet, die vor Moders Bryst og opholdes ved hendes Pleie , Omhu og Flid . Dette er at stille det Gode frem for Lyset og ikke alene se hen til det Onde , men ogsaa hente Glcede af Guds Gåver og Velsignelse og dermed tilhylle og bedcekke Straffen , Urenheden , Smerterne og Angsten . Men tun fromme Christne vide og gjore dette ; de give <b>Kvinderne</b> , som det svagere Redstab , deres efterdi de se , at ogsaa de have Del i det evige Liv og ere Medarvinger til Himmeriges Rige . Derncest se de ogsaa hen til , at de ere vrydede og benaadede med Guds Velsignelse og Moderceren , og at vi Alle af dem undfanges , fodes , nceres og opdrages . 201. leg selv foler ofte stor Glcede og Forundring ved at fe , hvorledes en <b>Kvindes</b> hele Legeme er stikket og dannet til at ncere og opdrage Born . Hvor godt tåger det sig ikke ud , endog for Smaaviger , at bcere Born ? Hvor pent og godt forståa ikke Modrene at lege og spoge med sine Born , foråt faa dem til at give sig tilfreds , nåar de grcede , eller at lcegge dem i Buggen ? Lad nu en Mand gjore Saadant , faa vil dn nok maatte sige , at han forftaaer sig lige saa godt paa det , som en Kamel paa at dandse ; saa ilde tåger det sig ud for ham , endog blot nåar han stal rore ved Barnet med en Finger . leg vil ikke tale om de andre Tjenester og den ovrige Pleie , som Bornene kun kunne faa af sin <b>Moder</b> . Hvo der nu ret vil betragte alt dette , han maa erkjende det for scerdeles Spor og Beviser paa Herrens Velsignelse , hvorved Gud viser , at han tåger sig af det <b>kvindelige</b> KjM og har det kjcert , selv om det er haardt straffet for Syndens Skyld . Og dette vcere nok om den forste Del af Forbcmdelsen . 202. Den anden Del angaaer den cegtefiabelige Forbindelse . Thi

<h4>2439</h4>Ugudelige og forlese Kirken . Derover blive Mange mismodige af Utaalmodighed, og ville ncesten forsage . 269. Men lad os efter Abrahams Exempel ogsaa bie vaa Herren og holde ud for ham . Thi han stal visselig komme , nåar vi tun forbliue i Taalmodighed , og gjore en gruelig Ende vaa Tyranniet . Men sin Kirke vil han opholde ; thi det er hans <b>Vis</b> , at han , som Esaias siger Kap . 54 , 7. 8. , forlader Sine et lidet Bievlit og skjuler sit Ansigt , men at han siden samler dem med stor Barmhjertighed . Men ligesom Kirken og de Fromme skulle opretholde og troste sig med dette Haab , saaledes stulle de , som det gaaer vel , og som plage Kirken , frygte for Straffen . Thi omend Herren forhaler den , faa vil han dog ikke evig holde inde dermed . Og endnu har der aldrig vceret nogen faa mcegtig Konge eller Stat , som ikke er bleven straffet og er ganet under for sit ugudelige Vcesens Skyld . Men nåar , hvorledes og paa hvilket Sted dette vil ste , det stulle fromme Christue overlade til Gud , faaledes som Abraham her gjor . 270. Meu her svorges , hvad dette var for en Plage , hvormed Herren flog Pharao og hans Hus ? Thi det viser Me Terten , ligesom Mer ikte , hvorledes og hvorved Pharao fik tilkjendegivet Aarsagen til Plagerne . Thi at Abraham ikke gjorde det , er sikkert , siden han jo havde bedraget Kongen ved at sige , at Sarar var hans Soster og Ne hans <b>Hustru</b> . Og det ser ud til , at Kongen var uskyldig deri . Thi han fratog ikke en anden Mand hans <b>Hustru</b> , men han tog En , som var ugift , og fom han heller ikke vilde misbruge som en Skjoge , men beholde som sin Wgtehustru . 271. Men Ipderne hader jeg , fordi de , nåar de tale om faadanne uvisse og tvivlsomme Ting , komme frem med idel taabeligt og urimeligt Snak , faaledes som de her sige , at Plagen bestod i den Sygdom , fom Lcegerne kalde Gonorrhoe , nemlig at den mandlige Sced af sig selv spildes . 272. Sidenefter , da Sara roves af Philisternes Konge , siger Texten 1 Moseb . 20 , 18. , at der fulgte en saadan Straf derpaa , at alle <b>Kvinders</b> Moderliv lukkedes , faaat de hverken kunde undfange eller fode . Noget Saadant , tcenker jeg , stede ogsaa her . Thi efterdi Kongen tåger en fremmed <b>Kvinde</b> , straffes han ogsaa med en Straf for <b>Kvinderne</b> , nemlig at en <b>Kvinde</b> ikke kan vcere <b>Kvinde</b> , det er , at hun ikke kan fpde , men maa lade Livsfrugten komme for Tiden og lide anden Nod ved Fodselen . Thi saa gaaer det scedvanligvis , at hvormed Nogen synder , dermed bliver han og plllget , som der staaer strevet i <b>Visdommens</b> Bog Kap . 11 , 17. 273. Og her faa atter Egypterne det Lov , at de nogenledes havde Guds Erkjendelse og forstode , at denne Plage ikke var et Tilfcelde eller en simpel og almindelig Ulykke , men at den var en Straf , sendt af Gud for en bestemt Synds Skyld . Thi dette er ikke en liden Plage ; men jeg mener , at blandt al Dodens Nod kan der ikke vcere noget sorgeligere Billede og grueligere Syn , end nåar en <b>Moder</b> don tilligemed sit Barn . 274. Da derfor dette oftere hcendte , efterat Sarai var bleven rovet , tcenkto » Kongen , dreven dertil af den Hellig-Aand , som rorte hans Hjerte , ester , hvad der vel var Aarsag til denne Plage . Og efterdi denne Plage var kommen ind i Kongens Gaard sammen med Sara , saaat <b>Kvinderne</b> ved Hoffet enten kom i stor Fare , nåar de skulde f > de , eller dode ved

<h4>3013</h4>134. Uden Frugt bliver Guds Ord aldrig ; derfor bliver ogsaa her den gjenstridige , stolte og ulydige Hagar ved Englens Prcediken omvendt , vender tilbage til sin Frue , og sinder sig taalmodig i hendes Magt over sig ; ja hun gjor Me det alene , men erkjender Guds Barmhjertighed, priser Gud og paakalder ham med et nyt Navn , foråt prise hans Velgjerning , fom han havde bevist hende . 135. Saaledes kalde vi i det nye Testamente Christum en Frelser af hans Gjerning , hvormed han har aabenbaret sig for os ; den Hellig- <b>Aand</b> kalde vi en Troster ; saa kalder Hagar Gud en Seer , saasom han havde seet til hendes Ddmygelse eller Elendighed . Og dette Exempel er os ogsaa nyttigt til Lcere , paadet Enhver i sit Kald maa lcere at erkjende Guds Velgjerninger , vcere tccknemlig derfor og prise dem ; ligesaa foråt vi taalmodig kunne taale Tugtelfe af dem , fom ere fatte over os , efterdi Gud har Velbehag i faadcm Taalmodighed og sender os Hjcelp . 136. Saaledes er Skriften , fom St . Paulus siger 2 Tim . 3 , 16. , nyttig til <b>Lcerdom</b> , og er Me en unyttig Grammatik , hvori man kives om Bogstaver , Tegn eller Ord . Thi ved dette Hagars Exempel belceres vi ogfaa om , at vi stulle give den Gud , fom har trostet os , 3 Oren , og sige : Herre , du har hjulpet mig af min Sygdom , du har bestaaret mig Eiendom , du har givet mig en from <b>Hustru</b> o . f . v. 137. Dette er Lovofferet og den velbehageligste Gudstjeneste , nemlig

<h4>3446</h4>- 96. Efterat nu Moses har bestrevet Gjcestebudet med sin Herlighed , hvis Lige aldrig har vceret under Solen , siden jo Gjcesterne ere Gud selv og hans Engle , fortceller han nu videre , hvad der var deres Samtale eller Prcediken ved Bordet , foråt der Me stal vcere nogen Mangel ved denne Vestrivelse , og foråt det stal blive bekjendt for hele Verden , at det ikke gik saaledes til ved dette Gjcestebud , som mellem Munkene , der maa tie stille ved Bordet . 97. Thi der er jo intet Fortrcedeligere eller mere Upassende og Übehageligt , end nåar det gaaer stiltiende til i et Gjcestebnd , hvor gode Venner tomme sammen . Thi Ord og Samtaler , der , som St . Paulus siger Col . 4 , 6. , ere yndige og lrndrede med Salt , er det rette Krydderi vaa Maden , nåar det ene Ord bringer det andet med sig og stjcerper det , og ilke alene Legemet bespises , ' men ogsaa Hjertet modtager en nyttig <b>Lcerdom</b> . Thi christelige Samtaler troste og vederkvcege Hjerterne , ovvcekke Tro , opftamme til Kjcerlighed og undervise os vaa mange Maader . Derfor lade vi de fortrcedelige og stille Munke fare , som anse sin Taushed for Hellighed og Gudstjeneste . 98. Nu da . Sara havde , som det lader til , tvivlet noget vaa den Forjcettelse , som Gud havde givet hende ovenfor i Kap . 17 ( V . 16.19 . ) , at hun skulde vcere den forjcettede Sceds <b>Moder</b> ; derfor sporger her Herren ester hende , for selv ved sit Ord at styrke hende i Troen . Thi dette er altid Guds Bestrcebelse at undervise de svage Hjerter med sin <b>Aand</b> , oplyse og styrke dem , og han fordommer eller forkaster dem ikke for deres Svagheds Skyld . 99. Derfor fvorger han her strax , hvor Sara er , og Abraham svarer kortelig : se , hun er i Paulunet . Naar et dovent Hjerte lceser dette korte Svar , lader det det fare , fom om det intet Synderligt havde

<h4>3659</h4>Tilfcelde , men Regler . Du er ilke Loth , er ikle Abraham , derfor stal du ikle gjsre efter , hvad Loth og Abraham have gjort . 86. Ligesom den oproreriste <b>Aand</b> Thomas Munzer foreholdt sine VMder Davids Exemvel , da han kjcemvede mod Syrerne , og be « d dem , efter dette Exemvel , at kjcempe mod Adelen . Dette er at gjore en enkelt Gjerning til en Ret og gjore et scereget Tilfcelde til Regel . Dette gaaer aldrig af uden Fare , fordi man , nåar man afviger fra den almindelige Regel , som er forestreven i Guds Ord , og bygger blot vaa en Gjerning , derved frister Gud . Hvo der derfor ikle ved en sceregen Befaling af Gud kaldes udenfor Guds Ords Regel til at vcere en Undermcmd , saaatsige, han holde sig til Regelen i Guds Ord , saa vil han ikke feile eller fare vild . Men Loth , Abraham og saadanne scerdeles Hellige ere Undermcend , hvis <b>Aand</b> ryktes udenfor Regelen , og vandre udenfor alle Exempler og Folg , eslutninger . Thi de have et scereget Kald og en HK Stilling , som du ikke har . 87. Derfor stal man lcere , at man , nåar saadanne Historier fortcelles, da ikke stal lcegge Vcegten vaa Exemplerne eller de enkelte Gjerninger, men vaa Ordenen og Regelen i Guds Ord . Hvad Loth har gjort , kommer dig ikke ved . Thi hvi iwmmer du en fremmed Svend ? Rom . 14 , 4. Men Guds Bud kommer dig ved , at du i Gudsfrygt stal opdrage <b>Hustru</b> og Born og vogte dig for at give dem Aarsag til at synde . Trceffer en Fare dig , og du kan ovholde dem i Tugt og 3 - Erbarhed , stal du ikke selv fore dem i Fare og udscette dem for Vancere eller uudstylde dig med Loths Exemvel . Thi du er ikke Loth , og efterdi du forlader Regelen i Guds Bud , frister du Gud . Thi dig er det ikke befalet at vige af fra Regelen og gjpre Andres Exemvler og underfulde Gjerninger efter . 88. Abraham ihjelstaaer fire Konger ; to Gange scetter han sin Hustrues og Kydsthed vaa Spil . Loth overgiver sine Dpttre til Skjcendsel . Men hvorfor de gjore dette , kommer ikke dig ved ; thi Gud har vaa denne Maade ved dem lagt sin <b>Visdom</b> og Magt for Dagen . 89. Derfor kan jeg nok forundre mig over saadanne Gjerninger , men eftergjore dem kan og stal jeg ikke . Ligesom vi undre os over , at Petrus gaaer vaa Havet , Matth . 14 , 29. , at Christus og Moses faste fyrgetyve Dage , Matth . 4 , 2. 2 Mos . 24 , 18. ; men ikke efterligne disse Gjerninger . Thi de Gjerninger af de Hellige , som udfores imod Guds almindelige Bud og Orden , ere Undergjerningar , som vi ikke kunne gjore dem efter . Men er det saadanne Gjerninger , som man ikke kan billige eller uudstylde , saasom at Loth besover sine Dottre , da er det Synder , som ikke stulle ste af os . Men nu er det ikke nyt eller uhort , at de Hellige falde og snuble . Syv Gange , siger Skriften Ordsv . 24 , 16. , tan en Retfcerdig falde . Thi efterdi de ligesaavel som vi have et syndigt Kjod , er det intet Under , om de undertiden falde ; men kan man uudstylde dem , stal man prise Gud derved . Saaledes er det min Mening , at Loth er en Undermcmd , ved hvem Gud vilde aabenbare sin <b>Visdom</b> og Magt . 99. Efterdi saaledes Skriften afverlende foreholder os sceregne Gjerninger , Rette , Exempler , Negler , Mircckler og guddommelige Bud eller Love , stal man af alt dette alene betragte , lcere og indprente Retten, Regelen og Loven . Naar en Svroglcerd opstiller denne Regel : vaa Latin ere alle Ord , som ende paa a . , af Huukjon , saa er Ordet ? c > Bin.a

<h4>3994</h4>141. Men her opstaaer nu . dette Sporgsmaal , hvorledes han kan sige , at Sara er hans Faders , Tharas , Datter ? Thi Moses siger jo tydeligvis ( Kap . 11 , 27. ) , at Thara havde tre Ssimer , Nachor , Hamn og Abraham , og at Loth var en SM af Haran , men at Loth var Saras Broder . Men om Sara siger han Me , at hun var Harans Datter , medmindre du vilde sige , at hun havde to Navne , og at hun ovenfor i Kap . 11. ( V . 29. ) kaldes lista . Men da dette er noget for urimeligt , er vel den ncermeste Vei , at vi mene , at hun var Harans Steddatter , efter hvem hun , da han dpde for sin Fader Thara , blev bragt i Tharas Hus og der avdraget , faaledes at hun altsaa kaldes Tharas Datter uden at vcere Harans kjedelige eller rette Datter , men antaget som Datter ved Enkens Giftermaal . Folge vi denne Mening , saa udelukkes allerede derved Talen om , hvorvidt Abraham kunde tåge sin Broders Datter til <b>Hustru</b> ; thi at cegte sin Broders Steddatter var ikte forbudt i Loven . 142. Hvad der i Texten lyder : den Tid Gud lod mig vanke hid og did af mm Faders Hus , betyder paa Hebraisk : den Tid Guder lode mig vanke o . s . v. Denne Text er flittig at betragte og vel at mcerke . Der er ellers almindeligt , at Flertalsordet Elohim scettes for Guds Navn ; men paa dette Sted scettes til Flertalsordet Elohim ogsaa Gjerningsordet i Flertal , saaat det paa Latin lyder teesruQt srrars . Ipderne bespotte og forfalske , som man ved , denne Text saaledes , at de foregive . Moses stal have talt saaledes af 2 Erefrygt for Guds Majestcet . Men vi skulle mene , at Moses her har havt netop de samme Tanker , som ogsaa i forste Kapitel af denne forste Bog V . 26 , hvor Gud siger : lader os gjsre et Menneske , nemlig for derved at <b>vise</b> , at der i den ene Guddom er flere Personer , nemlig Faderens , SjMnens og den Hellig-Acmds Person . Og dette have vi ikke opdigtet , men tåge denne <b>Visdom</b> af Guds Ord , som jo er aabenbart og klart , saa vi ikke stulle forfalske det , saaledes fom loderne gjore , iscer fordi Skriften saa fljMt stemmer overens , og det nye Testamente saa klarlig beviser det Samme .

<h4>4216</h4>de ere prydede , og Paven og hans Kardinaler ere vel ogsaa saare <b>vise</b> Mend , kan jeg tcenke . Saaledes vare ogsaa Plato , Cicero , Sokrates store Mend , men ere derfor ikke Kirken . Thi de have ikle denne vesentlige Forstjel , ved hvilken Kirken adstilles fra Verden , nemlig Forjettelsen. Derfor indromme vi vel Paven og hans Tilhcengere , at der hos dem er <b>Visdom</b> og stor Fornemhed , desuden , at de ogsaa have den uafbrudte Rekkefolge og Embedet ; men derfor ere de endnu ikke Kirken . Thi de mangle endnu den vesentlige Forstjel , nemlig at man maa tro Evangeliet . Men de , som ikke tro Evangeliet , ere Me Kirken , henhore heller ikke til Christi Rige . 188. Derfor , kjcere Junker Pave , maa dit Pavedømme forst dodes og dn selv ganske tilintetgjores for Gud , ellers vil du aldrig blive selv det ringeste Lem paa Kirken . Dette er vel haarde Ord og utaalelige for Papisterne . Men det gjorde ogsaa Ismael ondt , at han maatte udstodes og frastilles fra Kirken og ansees for et saadant Lem , som var afstaaret fra Kirken . Os simple Mennester kan der heller ikke times noget Haardere eller Vesvcerligere , end nåar man saaledes beroves alt Haab og Trost . 189. Denne Anfcegtelfe plager ogsaa mig ofte , at jeg ser mig om efter gode Gjerninger , som jeg kunde forlade mig paa , at jeg nemlig har lcert og prcediket meget , tjent mange Folk og gjort dem Godt , dertil lidt megen Uret uden min Skyld ; men nåar det rigtig kommer til Kamp og Strid , da foler jeg nok , at alt dette er Intet , og drives derhen , at jeg med David maa bekjende Pf . 32 , 5. : Herre , jeg er , lntet uden en arm Synder . Ligeledes nåar han siger Ps . 116 , 11. : jeg sagde i min Angst : hvert Menneske er en Lsguer . Ligeledes Ps . 143 , 2. : gan Me i Rette med din Tjener . 199. Men med dette Haab ovreiser jeg mig alene , at jeg ser , at der i Evangeliet er forjcettet Trost for dem , hvem deres Synd trykker og crngster , og Haab for de arme , forsagte Samvittigheder ; dertil at Himlen er forjcettet dem , fom ere nedforte i Helvede . Og dette er et sikkert Bevis for dette Haab , at Guds Son , uden at vi forud vidste om det , har offret sig selv til sin Fader for os paa Korfets Trce . 191. Ere nu Folk paa denne Maade forst ydmygede og bragte til Fortvivlelse , og , idet de forsage om sig selv , ogsaa begynde at fatte Mod for Christi Skyld , faa blive de Guds Born og Arvinger . Men man sinder Mange , fom ikke ville ydmyges , men endnu mene at hcevne sig , og knurre mod dem , fom revse dem ; disfe ere dobbelt forhcerdede og forstokkede . 192. Naar du derfor foler , at du ydmyges , faa lceg dig ned til din himmelske Faders Fodder og sig : o kjcere Herre , nåar du handter saaledes med mig , vil jeg med Taalmodighed lide det , og bekjender , at jeg vel har fortjent en meget haardere Straf ; derfor forbarm dig over mig ! vil du ikke , at jeg stal vcere Arving , saa gjor det dog saaledes med mig , at jeg kan forblive Tjener ; ja , fom den kcmanceiste <b>Kvinde</b> siger Matth . 15 , 27. , jeg vil ikke vcegre mig ved at vcerr en Hund i dit Hus , saa jeg ialfald kau spise de Brodsmuler , som tilfeldigvis falde til Jorden og nedtredes . Dn er mig efter Retten Intetfomhelst skyldig ; derfor holder jeg mig til din Naade og Barmhjertighed . 193. Dette er den rette Vei , ved hvilken vi komme til Naade og Salighed ; men faa ere de , fom hore det , og endnu ferre de , som adlyde det , tro og antage det . loderne opramse daglig i sin Bon disse

<h4>4367</h4>31. Men hvad mener du vel Abraham Me i sit Hjerte ? thi han havde jo Kjod og Blod og var , som jeg ofte har sagt , ingen haard Mand . Men det har vel ogsaa foroget hans Smerte , at han ikte turde aabenbare Sagen for Nogen , ellers vilde Enhver have fraraadet ham det , og den store Hob af dem , som saaledes havde fraraadet det , vilde maaste ogsaa have gjort Indtryk paa ham . Derfor begiver han sig alene paa Veien med nogle Tjenere og sin Son . Det er sandelig et mcegtigt , tungt Bud , langt haardere , end vi kunue tceuke os det ; og dog er alligevel det fuldt af Trost , at Texten klarlig siger , at Gud tun gjorde det foråt forsoge ham . Havde ogsaa Abraham vidst dette , da havde hau nceret desto mindre Bekymring ; men nu er han ganske nedsunken i disse Tanker , at hans Son virkelig maa offres og drcebes , Forjcettelsen faaer saa opfyldes nåar og hvorledes den vil . 32. Herren bestemmer og ncevner ogsaa et scereget Sted dertil , udenfor hans Hus , i Moria Laud . Men dette Land ligger henimod otte Mile fra Beersaba , om ellers Geogravherne trceffe det ret ; det er ncesten tre Dagsreiser . Thi han drog med et Wsel , som maatte bcere Veden og Levnetsmidler ; derfor kom de kun langsomt afsted . 33. Men Moria er , som der staner 2 Kronik . 3 , 1. , det Bjerg , som laa ncer ved Bjerget Zion , paa hvilket Salomo byggede Templet , midt i Byen Jerusalem , mod Norden , der hvor Staden skraaner nedad Bjerget . Men dette Bjerg gav hele Landet Navn , saaat det blev kaldt Moria Land . Og Tyrkerne kalde endnu den Dag idag Bjerget med dette Navn , og holde det i stor 3 - Ere . Saaledes hcender det endnu ofte , at et Bjerg eller enkelt Sted giver et helt Land Navn , ligesom jo hele Sachsen har sit Navn af Slottet Sachfenburg . Paa samme Maade blev ogsaa dengang hele Landet af Bjerget Moria kaldt Moria Laud , i hvilket Jerusalem laa , som dengang kaldtes Salem eller labus , og hvorover Patriarken Sem regjerede . 34. Men Ordet Moria skrives paa Hebraisk med ij . Og Hebrceerne tale meget udforlig om dette Ords Vcesen . Hieronynms har her ovfattet det som et Fcellesnavn : gaa til Synets Land . Men denne Afledning af Ordet forkastes af alle Lcererne , saasom den ikke ret stemmer med Sprogets Regler . Andre sige , at Ordet kommer af Ordet Myrrhll , og at Morijah stal vcere saameget som Mirrha men Dominus , d . e . Herren er min Myrrha , fordi han der nemlig vil have Myrrha , det er sin Gudstjeneste ; thi der voxede paa det Sted Myrrha og Nogelse , som man brugte til Rogelsesoffer . Og dette trceffer til ; thi Ordet <b>Mor</b> betyder Myrrha ; Bogstavet j er det tilhcengte Pronomen , min , og jah er et af de Ord , der udtrykke Guds Navn . 35. Men om jeg end ikke just vil angribe denne Afledning og Udlceggelse af Ordet Moria , saa folger jeg den dog heller ikke , fornem- af den Aarfag , at loderne efter stu scedvanlige <b>Vis</b> kuu have autaget den ydre Gudstjeneste med Rogelsesofferet , men ikke forståa den fande og indvortes Gudstjeneste , heller ikke holde den . 36. Derfor tykkes deres Mening mig at vcere mere passende , som sige , at Moria kommer af Ordet Jara , fom betyder at lcere , hvoraf ogfaa Loven kaldes Thorah , da den jo ogfaa kan kaldes en Undervisning eller <b>Lcerdom</b> . Saaat altsaa Meningen af dette Ord er : Herren , som underviser , fordi Gud nemlig lcerer og hores paa dette Bjerg , som Propheterne sige : af Zion stal udgaa en Lov , og Herrens Ord af Jerusalem, Es . 2. 8. Denne Afledning tykkes mig bedre og mere pasfende

<h4>4600</h4>Liv . Tvivler du der endnn angaaende denne <b>Kvinde</b> , om hun ogsaa er din Wgtehustru , saa er hun Me din Wgtehustru , men en Skjage . Thi ved enhver Netfcerdighed , Me alene de Christnes , men ogsaa Hedningernes og Tyrkernes , maa der vcere Vished og Overbevisning . En Tyrk er <b>vis</b> pcm sin <b>Hustru</b> , sit Hus , sine Tjenestefolk og sin Keiser 0. s . v. Saaledes maa ogsaa en Christen slutte og vcere <b>vis</b> paa , at hans Gjerninger behage Gud og ere gode , hvad enten de ste i Kirken , i det huslige eller borgerlige Liv . Og det er et sandt Bedrageri , nåar Modstanderne lcere , man stal gjore gode Gjerninger og dog alligevel ikke forlade sig paa dem . Thi hvoraf " komme ellers Propheternes Prcedikener: de tilbede sine Homders Gjerninger , Jer . 1 , 16. , og flere saadanne, uden alene deraf , at Alle , som ville blive retfcerdige ved sine Gjerninger , i Virkeligheden ere afgudiste Mennester ? Thi de kunne ikke bare sig foråt forlade sig paa Gjerningerne . Hvorfor forrettede jeg saa tungt Arbeide i Klostret ? Hvorfor pinte og plagede jeg der saaledes mit Legeme med Faste , Vaagen og Frost ? Fordi jeg vilde vcere sikker og forvisset om , at jeg ved saadanne Gjerninger skulde faa Syndernes Forladelse . 298. Derfor skulde du svare en Papist saaledes : du modsiger dig selv , og hvad du foregwer i din Lcere , strider stik imod hinanden . Du lcerer Gjerningernes Netfcerdighed , og vil dog ikke have , at man stal forlade sig derpaa , da jo dog enhver Netfcerdighed nødvendigvis i sig indeslutter Tillid , men Mistilliden er det Bevis , hvoraf man kan flutte , at der ingen Netfcerdighed er tilstede . Men fcetter du din Lid til Gjerninger , da er du jo en Afgudstjener . 299. Er nu ikke det at lcere faadanne Ting , som stride stik imod hinanden ? Thi hvo vilde her ikke se , at de slaa sig selv slemt paa Aret ? Retfcerdiggjorelsen ved Troen indromme de snart , snart ncegte de den atter , vistnok ikke i Ord , men i Gjerning , efterdi de sige , at Troen i Forening med Gjerningerne gjor retfcerdig ; og dog ville de have Tilliden til Gjerningerne udelukket . 399. Og hvad de forhen unyttig have opdigtet og foregivet om Mennestescetninger og Lovens Gjerninger , forkastes ogsaa ved deres egen Dom og Bekjendelse , og gjcelder slet Intet . Men efterat de have grebet og faaet fat paa det Ord Tro , saaledes at de ncegte og sige , at Troen alene ikke er nok til Salighed , men indflette Gjerningernes Fortjeneste tillige , miste de nu ogsaa Troen tilligemed Gjerningerne . 391. Men derfor anfore de Texten Luc . 17 , 16. , hvor Christus siger : imar I have gjort alle Ting , som Edcr ere bcsalcde , da siger : vi ere unyttige Tjenere ; thi vi gjorde det , som vi vare skyldige at gjore . Dermed forfvare de sin Lcere , at man ikke stal forlade sig paa Gjerningerne, men stode derved atter skammelig an . Thi netop dette , som de mene tjener dem til at bekrcefte deres Lcere , svcekker og omstoder den ganste . Thi Christus forkaster og fordommer der ikke alene Fortrostningen til Gjerninger , men ogsaa al Netfcerdighed og Fortjeneste ved dem . Thi stal man sige , at Gjerningerne Intet nytte , saa folger jo deraf med Nodvendighed , at de ikke ere Retfcerdigheden , heller ikke kunne hjcelpe til det evige Liv , men ere ganske og aldeles udygtige og Intet . 392. Derfor forkastes ganste al Tillid , Netfcerdighed , <b>Visdom</b> og hvad Mere der er af Gjerninger , og det er falskt og Logn , at vi blive retfcerdige ved Tro og Gjerninger i Forening ; mensen velsignede Sced

<h4>4914</h4>142. Men det er en herlig , stor Ros , og Englene prises HM derved , at de lade sig bruge til saadanne simple Tjenester , der , som det tykkes os , ikke passe sig for dem , og vi maa , ester vor medfodte Hoffcerdighed og Stolthed , vel takke Gud for , at vi ilke ere stabte til Engle ; thi vi vilde ikke kunne finde os i saadan simpel , foragtelig Tjeneste , og vilde Alle fulgt Lucifer efter . Thi er ikke det en foragtelig og uren Ting , at tilfore Isak en Brud ? Thi fornden at den menne- Mige Natur er fordcervet og gruelig stjcemmet ved Synden og dens Straf , saa har det <b>kvindelige</b> Kjon endnu en synderlig Modgang og et scereget Kors at bcere fremfor Mcendene ; skulde da en saadan hellig Engel lade sig sende til en <b>Kvinde</b> ? Det vil jo dog siet Me passe sig for Englenes Majestcet . 143. Saaledes pleie vi at gjore , vi som dog ikke ere And et end Skarn , Stank og Orme ; nåar Nogle overgaa Andre i sceregne Gåver , saasom at de ere veltalende , <b>vise</b> og forstandige , ja endog have mere Lykke og Gods end Andre , hjcelp Gud , hvorledes hovmode ikke da saadanne Folk sig over de Andre , som ikke have disse Gåver , medens de dog i Henseende til Synd og allehaande Ulykke og Jammer ere dem ganske lige ? Saaledes mente ogsaa Djcevelen , det ikke vilde stikke sig for Englenes Majestcet at tjene Rebekka , og det uden at Abraham bad derom , men tun onstede det , idet han sagde til sin Tjener : drag hen . Herrens Engel vil vcere med mig ; og det tykkes jo ogsaa at vcere en saare billig Aarsag til at blive heftig og forlornes . 144. Men heraf fer man , hvad der er de gode Engles Art og Natur ; hvorledes dette er en saa ydmyg , kjcerlig og venlig Natur , som ikke tykkes sig at vcere for hoi og stor til at tjene arme Syndere , baade Mcend og Kviuder . Thi de ere fulde af Lys , Guds Erkjendelse og <b>Visdom</b> og af guddommelig Godhed . Hvad Gud befaler , forståa de derfor strår er saare herligt og godt , fordi det saa behager Gud . 145. Saaledes vcegrer Englen Gabriel sig ikke for at fendes til en Jomfru , som er langt ringere end ham , og bringe hende det Budstab, som var ham befalet af Gud . Thi han ved , at dette behager Gud vel ; det vcere nn en sceregen eller almindelig Befaling , derom fporger han ikke . Men de onde Acmder foragte ikke alene saadanne Tjenester , men Herren og Gud selv dertil . 146. Men efterdi vi nu forståa , at vi nyoe de hellige Engles Samfund , som de have med os , dertil deres Beskyttelse og Venstab , saa lader os takke Gud derfor og lader os efterfolge de hellige Engles Dyd og Kjcerlighedsgjerninger , og vcere venlige , gode og tjenstagtige mod hinanden indbyrdes . Intet Menneske er saa venligt og saa redebon og fccrdigt til allehaande Tjeneste og Velgjerning , som Englene ere . Men dette tro vi ikke , tilskrive heller ikke Englene disse Dyder . Dog er det vist , at Rebekka og Isak i Engelen havde en trofastere Ven og Vogter , end i Tjeneren og sine Slcegtninge . At Tjeneren var from og trofast , vidste de vel , men det var Intet imod Engelens Trofasthed og Gunst . 147. Derfor stulle vi lcere , at vore bedste og trofasteste Venner ere usynlige , nemlig de kjcere Engle , fom i Trofasthed og Kjcerlighed , dertil i allehaande Tjenstagtighed og sandi Venstab langt overgaa de synlige Venner , som vi have , ligesom paa den anden Side ogsaa de onde Engle og Djcevelen , fom ere ufynlige , ere os langt storre og fordcerveligere Fiender , end de legemlige og fynlige . Al Ulykke og hvad

<h4>4980</h4>Moses gjor saamange Ord om saadanne ringe Ting , da han dog ovenfor talte saa saare kort om langt hoiere Ting . Men det er ingen Tvivl om , at den Hellig-Aand har villei have , at dette stal skrives os til <b>Lcerdom</b> . Thi i den hellige Skrift foreholdes os Intet , som er ringe eller forgjceves ; men Alt , hvadsomhelst er strevet , er strevet os til <b>Lcerdom</b> , Nvm . 15 , 4. Thi Gud vil erkjendes i alle Ting , det vcere store eller smaa . 216. Ovenfor bestrev han med korte Ord , hvorledes Sodoma opbrcendtes, Kap . 19 , 24. , og Fristelsen , at Abraham skulde offre sin Son , Kap . 22 , 2. , efterdi det er hoie Ting , som et Menneske ikke lettelig forstaaer , hvis han ikke er <b>aandelig</b> ; men hvad der udvortes angaaer Legemet , forståa vi nogenledes lettere ; derfor taler Gud vidtloftig med os om saadanne Ting . Derfor stal det ikke fortrnde os at lcefe eller betcenke disse Ting , som beskrives saa udforlig i dette Kapitel . Og hvorvel her Artikelen om Menneskets Retfcerdiggjorelse ikke lceres , saa behandles her dog alligevel Lceren om stjonne , oerbare Sceder , ligesom der nu folger Exempler paa Venlighed , Tjenstvillighed og Godhed mod Fremmede . Og vi ere ved dette Kapitel indbudne til Rebekkas Bryllup ; derfor lader os hore Brudesangen , Ungdommen til Exemvel , paadet den kan lcere at have hcederlige Tanker om Brylluper og tillige om Mand og <b>Hustru</b> , hvilke Ting allesammen foragtes hos Hedningerne, som man kan se i Digternes Skrifter , bande de grceste og de latinske . Thi de se alene hen til Kjodet og forhaane og vancere derved Gud , Skaberen . Disse stulle vi holde den hellige Skrifts Text for Ncefen . 217. Det hebraiske Ord Nezem forekommer ofte i den hellige Skrift , og jeg troer , at jeg ved flittig Ovmcerksomhed har fundet dets rette og egentlige Betydning . I den latinske Text staaer Inauris , hvilket betnder en som hcenges i Vrene , hvilket omtales i Beretningerne om Morianerne og hos hvilke dette Smykke var almindeligt . Men det er endnu itte Ordets rette Mening , men det betyder egentlig , som Hierollymus ogsaa troer , et Pandebllllnd , som rcekker fra det ene Bre til det andet . Vore unge <b>Piger</b> bruge istedetfor dette Smykke Perler og gyldne Snorer ; men hint var purt Guld .

<h4>5057</h4>289. Derfor er Livet det hpieste Gode , hvilket man har af det <b>kvindelige</b> KjM , hvilket alene bencegter og gjendriver Skjceldsordene hos luvenal og Andre , som have forhaanet denne Stand og det <b>kvindelige</b> Kjsn . Skal man skaffe en By Bprn og Borgere , saa er det fornpdeni , at Folk gifte sig med hinanden ; og alt dette stemmer godt med det Navn , fom vor fyrste Fader gav sin <b>Hustru</b> , idet han kaldte hende Eva , det er de Levendes <b>Moder</b> , som fsder Be » rn , der sidenefter forvalte det borgerlige Liv , styre Kirken og Huset , lcere at erkjende Gud og blive salige i det evige Liv . 281. Vistnok have ogsaa Patriarkerne Mt og erfaret Uleilighed og Besvcer af 3 - Egtestanden . Abraham , Loth og de Andre vare Fremmede, reiste omkring og led megen Jammer og stor Modgcmg ; men det taalte de med Taalmodighed og tcenkte paa denne store Trpst , at Eva er de Levendes <b>Moder</b> , som fsder og opdrager Bsrn . 182. Derfor maa man kjcempe med denne Wgtestandens Msie , Uleilighed og Besvcer , eftersom vi altid have den ncervcerende Ulykte mest for Vie og Sind . Thi vi ere endnu ikke i det salige Liv , paa hvilket vi vente , men ere i et saadant Liv , i hvilket vi kjcempe og maa give os i Kast med megen Ulykke , hvilket vi Altsammen skulle lcere at tildcekke med Erkjeudelsen af Skabningen og Guds Ordning , som er god og guddommelig , og Ungdommen skal vcenne sig til at have rette og chriftelige Meninger om det <b>kvindelige</b> KjM og ZSgtestcmoen . 283. Ja , siger du , men al Verden og mange <b>vise</b> Folk foragte dog denne Stand og ere bange for den ? Svar : just derved give de sin Daarlighed tilkjende , efterdi de lun give Agt paa Skrpbelighederne, men ikke ovlukke Vinene , faaat de kunne fe Modervcerdigheden , som tilintetgjsr alt Onde og al Besvcer . Desuden priser og hcedrer Gud selv 2 Egtestanden i det fjerde Bud , hvor han siger 2 Mos . 20 , 12. : hcedre din Fader og din <b>Moder</b> . Men Alt , hvad han har indstiftet og forordnet i sine Bud , stal ingenlunde bespottes eller udskjceldes. 284. Hvo der kan afholde sig og leve kydst , lad ham afholde sig ; men alligevel stal han tcenke og tale hcederlig om og Modervcerdigheden; thi de ere Guds Velsignelse og Ordning , hvorved Gud opholder hele Mennesteslcegten i dette Liv . Dette har jeg villei gjentåge om Mgtestandens Ros . Thi Rebekka er Anledningen til , at den Hellig-Aand bruger saa mange Ord om og Forlovelfen , til <b>Lcerdom</b> og Exempel just for os .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009082103022'target=_>, 1859, Christelig Psalmebog</a><h4>1908</h4>33 ? . Wuds igjenfodte , nye , levende Sjele ! <b>Moder</b> vor Abba i syngende Flok , Herrens nye Slabningcrs listige Glcede Kan i hans Lovsang ei ove sig nok . Lader dog hver stg for Faderen hore , Hvem der hans Naade kan deiligst kundgjore . 2. Allerlivsaligste , mildeste Fader , Havde vi Thronernes Stemmer i Fryd , Havde vi Englenes Miner og Lader , Havde vi Cherubims Vinger og Lyd , Aldrig vi Alle tilhobe formaa ' de Notsom at prise den mindste din Naade . 3. Onde vi vare og dode som Stene , Haaroe i Hjertet som Marmor og Staal , Det var den Mcegtiges Styrke alene , Som os fil reddet , og naa ' de sit <b>Maal</b> , Lod os ved Aanden i Ordet fornemme Frelserens levendegjorende Stemme . 4. At ! hvad vi bleve da soelsom tilmode , Nu vi ret kjendte Fordervelsens Stand , Da vi tilforn ei Andet forstode , End at os feilede ikte et Gran , Stinkente i vor Begjcerligheds Grave , Da vi formeente dog Livet at have . 5. Naar vi betcentte hvad Faderen tcenkte , Da han begyndte at drage os ud , Da han os Livet imidlertid stjcentte , Smykkede <b>Sjelen</b> til Frelserens Brud , At han os elst ' de , os Onde saa saare , Maa vi henslyde i Kjcerligheds Tåare . 6. Syntes end <b>Sjelen</b> at smage Guds Vrede , Jamrede saare i Fodselens Nod , Det var i Sandhed Guds Kjarligheds Hede Den , som os haver til Himmelen sod ' , Maatte og Hjertet sin Jammer dog kjende , Forend det vilde til Korset sig vende .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100603020'target=_>, 1850, Det Nye Testament med Forklaringer, Indledninger, en Harmoni for de fire Evangelier, en Tidstavle for Apostelhistorien og et Sag-Register</a><h4>402</h4>41. Skjod . " Mcendene af Ninive stulle opstaa i Dommeu mod denne Slcegt , og fordomme den ; thi de omvendte sig ved Jonas ' Prcedi-42. Ken ; og se , her er mere end Jonas . , " Dronningen fra Sonden stal opstaa i Dommen mod denne Slcegt , og fordomme den ; thi hun kom fra Jordens Ender , for at hore Salomons <b>Visdom</b> ; og 43. se , her er mere end Salomon . " Men nåar den urene <b>Aand</b> er udfaren af Mennesket , vandrer han igjennem torre Steder , soger 44. Hvile , og sinder den ikke . " Da siger han : jeg vil vende om til mit Hus , som jeg gik ud af ; og nåar han kommer , sinder han 45. det ledigt , feiet og prydet . " Saa gaar han hen , og tager syv andre Aander tillige med sig , som ere vcerre end han selv , og , nåar de ere komne derind , bo de der ; og det Sidste bliver vcerre med det samme Menneste , end det Forste : saaledes stal det og gaa denne onde Slcegt . 46. IV . Men der han endnu talede til Folket , se , da stode hans <b>Moder</b> og hans Brodre udenfor , og begjcerede at tale med Dam .

<h4>882</h4>16. V . " Men der han vandrede vet » den Galilceiste Ss , saa han Simon og hans Broder Andreas , der kastede Garn i Soen ; — 17. thi de vare Fiskere . ' ^ Og Jesus sagde til dem : folger efter mig , 18. saa vil jeg gjore edcr til Menncste-Fistere . " Og dc forlode strax deres 19. Garn , og fulgte ham . " Og da han gik lidet frem derfra , saa han og lakobus , Zebedcei ( Son ) , og hans Broder , Johannes , der bodte 20. deres Garn i Baaden . " Og han kaldte strår ad dem ; og de forlode deres Fader , Zcbedceus , i Naaden med Leiesvendene , og fulgte efter ham . 21. VI . " Og de gik ind i Capernaum ; og strax om Sabbaten gik 22. han ind i Synagogen , og lcerte . " Og de forundrede fig saare over hans <b>Lcerdom</b> ; thi han lcerte dem som den , der havde Myndighed , og ikke som de Skriftkloge . 23. VII . " Og der var et Menneste i deres Synagoge med en uren 24. <b>Aand</b> , og han raabte hoit , " Og sagde : ak ! hvad have vi med dig at staffe , Jesu af Nazareth ! er dn kommen for at fordcervc os ? 25. jeg kjender dig , hvo du er , den Onds Hellige . " Og lesus truede 26. ham , og sagde : ti , og far ud af ham ! " Og den urene <b>Aand</b> sted 27. ham , og raabte mcd stor Nost , og for ud af ham . " Og de bleve alle forfcerdede , saa at de bcsfturgte sig med hverandre , og sagde : hvad er dette ? hvad er denne for en ny <b>Lcerdom</b> ? thi han byder 28. og de urene Aander med Magt , og de lyde ham . " Men hans Nygte udkom strax i alt det omkringliggende Land i Galilcea . 29. VIII . " Og de gik strax no af Synagogen , og kom i Simons 30. og And / rcce Hus , med lakobus og Johannes . " Men Simons Hnstrus <b>Moder</b> laa , og havde Feber ; og strax talte de til ham om 31. hende . " Og han gik til hende , tog fat ftaa hendes Haand , og reiste hende op , og Feberen forlod hende strax ; og hun tjente dem . 32. " Men der det var blevet Aften , der Solen var nedgangen , forte 33. de Alle , som havde ondt , og de Besatte til ham . " Og den ganste 34. Stad var forsamlet for Doren . " Og han helbredede Mange , som

<h4>1348</h4>som da I , som cre onde , vide at give eders Born gode Gaver , hvor meget mere stal Faderen , som er af Himmclcn , give dem dcn Hellig-Aand , forn ham bcde ? 14. 11. „ " Og han drev cn Djcrvcl ud , og den var stum ; men det „ stcde , der Djcevelen var udfaren , talede den Stumme ' , og Folkct 15. „ forundrede sig . " Men Nogle af dem sagde : han nddriver Djcevle 16. „ ved Beclzcbnl , Djcevlenes Overste . " Men Andre fristede ham , og 17. „ begjcercde et Tegn af ham fra Himmelen . " Men der han fornam „ dcres Tanker , sagde han til dem : hvert Rige , som cr splidagtigt „ med sig selv , blivcr ode , og et Hus , ( som er splidagtigt ) med et 18. „ andet , falder . " Men cr og Satanas bleven splidcigtig med sig sclv , „ hvorlcdes sial hans Nige da blive bestandigt ? thi I sige , jeg ud-19. „ drivcr Djcevle ved Veelzcbul . " Men om jeg uddriver Djcevle ved „ Bcelzebul , ved hvcm uddrive da edcrs Born dem ? derfor stulle de 20. „ vcerc edcrs Dommcrc . " Mcn dersom jeg uddriver Djcevle vcd 21. „ Guds Finger , da cr jo Guds Nige kommet til edcr . " Naar dcn „ Stcrrke bevcebnct bevogtcr sit eget Pallads , blivcr det , han haver , 22. „ med Fred . nåar en Stcerkere , end han , kommcr over ham , „ og overvinder ham , da borttager han hans fulde Harnisk , som han 23. „ forlod sig paa , og uddclcr hans Rov . " Hvo , som ikke er mcd „ mig , er imod mig , og hvo , som ikke sanker med mig , adsprcder . 24. „ " Naar den urene <b>Aand</b> udfarer af Mennesket , vandrer han igjennem torre Steder , og soger Hvile ; og nåar han ikke finder dcn , „ da siger han : jeg vil vende om til mit Hus , som jeg gik ud af . 25.26 . " Og nåar han kommcr , findcr han dct fciet og prydet . " Da „ gaar han bort , og tager syv andre Aander til sig , som cre vcerre „ end han selv , og nåar de komme ind , bo de dcr ; og det Sidste bli-27. „ vcr varre mcd dct Menneske , end det Forste . " Men det begav sig , „ der han sagde disse Ting , oploftcde en <b>Qvinde</b> af Folkct Nostcu , „ og sagde til ham : saligt cr dct Liv , som bar dig , og de Bryster , 28. „ som du diede . " Mcn han sagde : ja , salige crc dc , som hore Guds 29. „ Ord , og bevare det " . " Men dcr Folkct forsamledes til ham , begyndte han at sige : dcnne Slcegt er ond , dcn begjerer et Tegn , og der sial intct Tcgn gives dcn , uden Propheten lonce Tegn . 30. " Thi ligcsom Jonas var de Niniviter et Tegn , saa sial og Men-31. nestets Son vcere for denne Slcegt . " Dronningen fra Sonden sial opstaa for Dommen mod Mcendene af dcnne Slcegt , og sial fordomme dem ; thi hun kom fra Jordens Ender , at hore Salo-32. mons <b>Visdom</b> ; og se , her cr mere end Salomon . " De Mcend af

<h4>1350</h4>V . 15. 16. Her er tre Slags Folk , Spottere , som forhaane ; Tv , ' v < lende ; Tegn Begjarende . — V . 27. Lyksalig er din <b>Moder</b> , udraaber i Be- undring en <b>Qvinde</b> , som sikkert bavde modtaget et dubt Indtrnk af Icsu Tales Magt og Sandhed ; men lesus v. 28 vil ikke forbausende Beundring , men sit Ords Optagelse i Hjertet , Tro , Forbedring , Helliggjorelse , thi deri bestaar Saligheden , deri den sande Forening med ham , som ikke er afhangig af noget Idre , ikke af legcmligt Slagtstab med ham smlgn . Matth . 12,47 — 50 ; ' saa lyksalig som denne ' Ovinde priser Jesu <b>Moder</b> at vare , kan Enhver , kunne Alle vare , uaar de kun optage i sig hans Ord , i Ordet bans <b>Aand</b> . — V . 29. 30 straffe den tomme Mirakclsogen ' , som miskjendcr tet storste Under , lesus selv , og bolter sig ved det Sanselige : — V . 31 straffes den last . vcerdige Ligegyldigbed imod Cbristus , som er vor <b>Visdom</b> 1 Kor . 1 , 30. Kol . 2 , 3 ; — V . ' 32 lafter lesus Übovfardighcden : alt dette er desto strafvardigere

<h4>1796</h4>gelser , vilde han dog fra sin S ! oe anvende ethvert mcnnesteligt Middel for at undgaa Efterstrabelscrne . — V . 2. Jesus dobte ikke selv , fordi Andre lettelige» kunde udfore denne Forretning smlgn . ogsaa 1 Kor . 1 , 17. — V . 4. Deri korteste Vei , tre Dagsreiser , forte ham gjennem Samaria , og han benyttede denne Vei , for at ' forkynde Ordet ogsaa for Samaritancrne , som viste sig overordentlig modtagetige for samme . Af v. 8 viser det sig , at Jesus selv ikte havde betraadt Staden ; v. 5 sichar ellers Sichem . 2. V . 7 — 9. Samtalens Oprindelsc . — V . 7. En <b>Qvinde</b> fra Samaria, fra Landstadet af dette Navn . — Den kjerre Herre ctttraaede Andet ; hnn torstede og hungrede ikte åtene tegemtigen , men meget mere aandctigcn , efter de arme fordomte Synderes Forlosning , til hvis Trost han var sendt . - ^ V . 8. Indkjobet af Spise hos Samaritancrne var Gjendrivelse af en jodist Fordom , ifolge hvilken Man ansaa saadan Spise for uren ; — v. 9 ligesom Man overhovcdet ansaa at Omgang med Samaritancrne for besmittende. 3. V . 10 — 15. Jesu Hensigt er ) at fremkalde <b>aandelig</b> Trang hos Ovinden , for sidenefter at kunne tilfredsstille den . — V . 10. Jesus vil opvcekke en Formodning om sin Heihed , derfor betegner han for <b>Qvindcn</b> hendes Sammenkomst mcd ham som en scerdeles Naadegave af Gud , og derfor taler han om den herlige Gave , som han , den Bedende , kunde skjente , nåar Man knn vitde bede ham derom , ( hvilket forudscettcr Attråa og Tillid i det bedende Hjerte ) , nemlig levende Vand , i dobbelt Forstand : Livets Vand og Kildcvand i Modsatning til det staaende Vand i Vandbehotdningcr . LigcsomKildevand atcnc formaar at stukke den reisende og forsmcegtcndePilegrims brandende Torst , saa er Herrens Gave , hans Ord , det rette Livets Vand , den sande Vedergvcegclse paa Pilegrimsreisen gjennem dette Liv . Vand forfriskar, frugtbcirgjor , giver Fremvcert , er uundvarligt , saa og Herrens Ord og hans Aands ' Stromme Si ^ ach 24 , 40 — 4 ' 4. — V . 11. 12. ' <b>Qvindcn</b> forstaar vel ikke lesnm , da hun opfatter hans Ord kun i egentlig Vetydning , men hendes Tittale : Herre ! og hendes Sporgsmaal tittjendegiver dog en Formodning om , at den Übetjcndte siger noget Stort om sig , kun ytrer hun v. 11 Tvivl , om ban ogsaa formaar at udrette noget saa Overordentligt , og v. 12 viser hun sig at ' vare hildet i forudfattede Meninger . Bronden , som endnu er til , er 105 Fod dyb og har kun fem Fod Vand . — V . 13. Dette Vand og ethvert Jordisk , jordisk ' <b>Visdom</b> og Lyst , tan ikke tilfredsstille Menneskets dybeste Lcengsel , idethoieste paa en kort Tid titsynetadende berolige den . — Derpaa skildrer Jesus v. 14 det Livsvcmds Fortrinlighcd , som

<h4>3548</h4>og straffer ikke det Onde , Synden hos en Ven , den tier , hvor den ssulde tale . Den sande Kjarlighed derimod hanger fast ved det Gode , om det end udoves af den varste Fiende , saa har den det dog kjart ; — v. 10 som Broderkjarlighed vare den inderlig , thi hjertelig kaldes egentlig den Kjarlighed , som Fader og <b>Moder</b> have til Born og Vrodre indbyrdcs ; den Ene forekomme den Anden med Wrbodighed » ' : Enhver vare ydmyg , holde den Anden hoit og i lEre , for Christi Skyld ; — v. 11 i dens Virksomhet » skildres den som ikke lunken , men ivrig ; som brcendende i Aanden , som ikke forst behover at tilskyndes udenfra / thi det er Aandens Art , at den ikke bliver trcet ; ved Stillcstand bliver den svag , ved Arbeide stark , men allerhelst bliver den brandende ved Forfolgelse og Modstand . — Tjener Herren , ifolge en anden Lasemaade : tjener Tiden smlgn . Prat » . 3 , 3. 4. , Kjcerligbeden skildres altsaa som <b>vis</b> i at benytte alle Tidsbegivenheder og Forholde i dette korte og fiygtige Liv . Dette er en ophoiet Låre imod de Gjcrningshcllige , som binde sig saaledes til Tiden , at den maa rette sig efter dem , og skikke sig i deres Vasen ; — v. 12 som taalmodig , og med ' Von , fuld af frimodigt Haab , udholdende i Trangsel , indtil den opnaar sit <b>Maal</b> . 2 V . 13 — 16. Denne hellige Ajcerligheds ytringer under de mangfoldige forstjcellige udvortes Forholde . — Den er velgjorende uden Egennytte Matth . 25 , 40 ; og herbergerer gjerne , Grass : jager efter Gjastevenssab, varer gjerne af Hjertet gjcestfrie , og giver Eders Gjcester Alt , hvad de behove ; — v. 14 mild endog imod Ficndtligsindcde , som ere Aarsag i Noden ; — v. 15 fuld af om Deltagelse og Medlidenhet » , stikket efter Andres Omstcendigheder ; vi skulle gjore os lige med Enhver , og stikke os i deres Sag , eftersom det gaar dem godt eller ilde ; er da Nogen glad , ssulle vi ikke se surt dertil , ligesom Hyklerne , som ville vare noget Scerdeles , og med deres utidige Alvor ville forcgive at vare alene <b>vise</b> og bcllige , og gjore Alle , som ere glade og ikke ligesom de se sure ud , til Narre og Syndere ; ' men deres Glade ssal behage os , nåar den ikke er Gud imod ; — v. 16 den er fredsommelig og langt fra al forfcengelig Selvgodbed , nedladendc i Idmyghcd .

<h4>3624</h4>ere mcerlclige iblandt Apostlene , som og for mig have vceret i 8. 9. Christo . " Hilser Amplias , min Elskelige i Herren . " Hilser Urbanus , vor Medarbeider i Christo , og Stcichvs , min Elskelige , il ) . " Hilser Aftelles , den Provede i Christo . Hilser dem , som ere af 11. Aristobuli ( Hus ) . " Hilser Hcrodiou , min Frcende . Hilser dem 12. af Narcissi ( Hns ) , som ere i Herren . " Hilser Tryphena , og Trvphosa, som have arbeidet i Herren . Hilser Pcrsis , den Elskelige , 13. som haver arbeidet Meget i Herren . " Hilser Nufus , den Udvalgte 14. i Herren , og hans og min <b>Moder</b> . " Hilser Asyncritus , Phlegon , 15. Hermas , Patrobas , Hennes , og Brodrene hos dem . " Hilser Philologns og lulias , Nereus og hans- Soster , og Olpmpa , og alle 16. de Hellige hos dem . " Hilser hverandre med et helligt Kys . Christi Menigheder hilse Eder . 17. 11. " Men jeg formaner Eder , Brodre ! at I give Agt paa dem , som volde Splid og Forargelser , tvertimod den <b>Lcerdom</b> , som I 18. have lcert ; og viger bort fra dem . " Thi Saadanne tjene ikke vor Herre Jesu Christo , men deres egen Bug ; og formedelst sode Ord 19. og smigrende Tale forfore de de Enfoldiges Hjerter . " Thi Eders Lydighed er bleven Alle bekjendt , derfor glceder jeg mig over Eder ; men jeg vil , - at I stulle vcere <b>vise</b> til det Gode , men enfoldige til 20. det Onde . * Men Fyedens Gud stal svart knuse Satcmas under Eders Fodder . Vor Herres Jesu Christi Naade vcere med Eder ! 21. Amen . " Timotheus , min Medarbeider , og Lucius , og Jason , og

<h4>3664</h4>Selvforncegtelse at bringe ethvert Offer , som Tid og Omstandighet » » maatte fordre , at Enhver for Himmelrigets Skyld maatte kunne give Afkalt endog . paa det Kjareste og Dyrebareste . Fra denne SyuSpunct udtaler han sig ogsaa v. 18 — 24 angaaende andre udvortes Forholde og Stillinger , og viser , hvorledes den Christne stal bedomme dem og forholde sig i dem . IV . Cap . B — lo . Undervisning angaaende den rette Brug af den christelige Frihed i Bisager . Af Modsatniuqen mellem lode- og Hedninge . Christne udviklede sig i Menigheden endnu en anden Strid angaaende Nydelsen af Afgudsofferkjod , og angaaende de Christnes Deltagelse i Gjastebude, som anrettedes af deres hedenske Slagtninge , Bekjendte og Venner , vet hvilke Man pleiede at nyde det Kjod , som var bleven tilovers as dc til Afgu . derne ofrede Dyr , og som ikke ogsaa var bleven brandt , og ved hvilke det ofte gik meget fraadsende til . loderne vare yderst samvittighcdsfulde i Valget af dcrci Naringsmidler , og hvor de levede sammen med Hedningene , bare de bestandig den storste Omsorg for , at de ikke siulde komme til offentlige » at hode noqctKjot eller forovrigt nyde Noget , som var bleven tilovers af Offerdyrene , thi de fryqtctc for derved at stode an imod Loven og gjore sig urene . De lodechristne havde ogsaa ofte denne pinlige TEngstelighed , og derfandtes ogsaa blandt de Hct » iin < gechristne anqsteliqe Gemytter , som ikke strår og aldeles ved deres Omvendelse kunde qjore sig fri fta den Forestilling , at de Guder , som dc bidtil havde tilbedet , dog maatte vare noget Virkelige , ; og da Alt , hvad som ikke stod i Forbindelse med Christus og bans Lysets og Naadens Rige , medrette forekom dem benhorentc til Morkets og Ondssabens Rige , saa ftygtede saadanne Hedningechristne for , vet Nydelsen af Afqudsofferkjod . paa en eller anden Maade at sattes i Berorclse eller i Forbindelse med Afguderne eller meqet mer med de ondc Aanders Rige , og derfor skyede ogsaa de Nydelsen af saadant Kjod 1 Kor . 8 , 1. 4 flg . De friere Sindede , for det meste til det paulinste Parti henhorcnde Cbristne , spottede ofte over denne deres Trosbrsdrcs pinlige Sueverbjertiqbed og anqsteliqe Samvittig , hedsfuldhed , og gik desaarsag ofte over til den motsatte Feil , at de uaqtsom til . lode stg altfor stor Frihed . Vistnok havde de rigtigen opfattet Paulus Gruvt . fatninger , hvad den blotte Erkjendelse anqaar , men de toge feil i dens Anvcn < dtlse . De erkjendte Afgudernes og Afgudsdvrkelsens Ugyldighet » , dc indsaa vel , at Nydelsen af saadantKjod hverken paa den ene eller paa den anden Maade kunde gjore Mennesket virkelig urent , derfor paastode de , at den Cbristne ikke er bunden til nogen Lov i saadanne udvortes , i sig selv ligcgyldige Ting , og nåar de tillode sig Nydelsen af Afqudsofferkjod eller den i Sandhed virkelig betankelige Deltagelse i hedenske Gjastebude endog i Afgudstcmvlerne 1 Kor . ' 8 , 10. , saa paaberacibte de stg til deres Rctfcerdigqjsrelse imod Svaqtroendc , som dadlede dette , og til Undssyldning for deres Deltaqelse endoq i uscedeliqe Gjestebude og Feste , dels ftaa deres Erkjendelse , ( Caft . 8 , i . Vi have Alle Kundssab ) dels paa deres christelige Frihedsret ( Cap . 6 , 12. 10 , 23. Jeg har Lov til Alt ) . Idet te nu slet ikke toge noget Hensyn paa deres Betcenkeliqhed , ' som vare svage i Troen , fordi de manglede Kjcerligbedens og <b>Visdommens</b> <b>Aand</b> , saa gav det ofte Aulet , ning til , at saadanne Svage , for ikke at blive spottede , gjorde eller node Noget , som var imod deres Overbevisning , hvorover de da siden solte beftiqe Samvittig , hedsnag . Paulus lader nu rigtiqnok de mere frisindede og starke Korinthers Paastand qjalde som den rette , at nemlig ingen bindende Lov kunde gives i saadanne udvortes Ting , men han viser dem fra Evanqeliets Standpunct , at , hvad i og for sig er tilladt , dog under bestemte Omstcendigheder ikke mere er tilladt , saasnart det strider unod Kjarlighedens Lov ; og denne Kjccr . liqhcdens Lov , ifolge bvilken bcstandiqt og af Alle kun det maa gjores , som er qavnliqt for Andres Sjelefrclse , og som ' mest tjener til Guds ofl ' Jesu Christi Forherligelse , fremholder han imod Misbrugen af den christelige Frihed , og for . lanqcr , at Udovelsen af den christelige Frihedsret skal stilles under Rjcerlighedens Veiledning . Den altid betankelige og anstodeliqe Deltaqelse ite navnte Gjastebude forbyder han aldeles Cap . 10. , og advarer for kjedelig Sit . kerbed ; Nydelse af Afgudsofferkjod tilsteder han under visse Tilfalde og Omstan . digheder Cap . 10 , 23 flg . V . Cap . 11. Modsatninqen mellem de ( engsteligere lodechristne og de mere frisindede Hedningechristne frembraqte ogsaa mange Misbrug ved de Christues al < mindelige Gudsdyrkelse , og dette strafferAposteleni 11. Cap . Forst v. I — l 6 mis < billisser han det , at de mere Frisindede ssjenncmbrode mangen gavnlig Scedvanei Skranke , idet <b>Qvinderne</b> , tverrimod den grcesse Landsssik , viste sig uden Slori Menighedens Forsamlinger , og vilde ligesom Mcendene tale offentligt ; derncest » . 17 — 34 straffer han alvorligen de Misbrug , som vare blevne fremherskende vet

<h4>3695</h4>V . 19. 20. Hvad som er strevet bos Jes . 29 , 14. , at Gud truer med at tilintetqjore alle <b>Vise</b> i Israel , for at dc , som vare forblindede i deres Hovmod, skulde blive qivne aldeles til Pris for deres Daarliqbcd , eller , almindeliqerc tåget , for at al forfanqeliq , tndbildt og formasteliq MennesteviSdom ( som Modsatninq til den quddommclige ) stal blive forkastet og tilintetgjort v. 19 ; — det sinder fuldkommen Anvendelse paa Evangeliet ; thi denne af Christus aabenbarede <b>Visdom</b> gjor al menneskelig <b>Visdom</b> til Daarlighed v. 20 , saa at alle <b>Vise</b> , bedcnste <b>Vise</b> , jodlsse Skriftkloge og alle Verdsligvise . spidsfindiqe Granskere og Disputmaqere , mcd al deres Kloqssab Intet qjcelde for Evangelium , fordi dette alene barSandbed ; eller har ikke Gud gjort osv. o : Gud ' har virkeligen fremstillet den ugudelige » sindede Verdens indbildte <b>Visdom</b> i al dens Daarligbed , thi den kunde jo tkke opdage Veien ti ' l , Sall ' qhed . — V . 21. Oq da Verden med sine <b>Vise</b> , ved Hjalp af al sin formente visdom ikke enqanq håvede sig til Erkjendelse af Guds Visdom, saadan som den nemlig aabenbarede stg i Skabrlsens og i hedens Stemme Rom . 1. 19 , 20. , men bekom istedetfor Sandhed kun tom Skinvisdom : saa behagede det Glid , saa var det bans naadige Raadslutning, ved en prcediken , som syntes daarlig , og som nedlod sig til den mennesseliqe Svaqbed , at udfore hvad hine Vife ikke formanede , at gsore Alle salige , som tro paa den , som troende vilde modtaqe denne Pradiken . Meninq : <b>Visdommen</b> fra Gud , aabenbaret i Evanqclium , opfyldcr altsaa bvad al anden <b>Visdom</b> ikke kan opfylde : den paaviser Veien til det evigt Liv . — V . 22 — 24. At denne sande , evanqelisse <b>Visdom</b> blcv miskjeudt af Verden , af loder og Hedninge , ( kun Saadanne gaves dentid ) v. 21 og f > rekom dem som Dacirliqhed , har ikke den samme Aarsag bos loder , som bes Graker , men en forssjcrllig Grund v. 22 , dels i deres Vcqjcerliqhed efttt , ' oincfaldende Mirakler , bos ' loderne Mattb . 12.38.39 . Cap . 16 , I — 4. Jol » . 2 , 18. Cap . 4 , 48. , omendssjondt Mirakler ikke bjalpe ugudelige Hjerter ; dels i Hang til orkeslose og frugteslose Granskninger ( Speculationer ) , bos Hetningene, hvorover Begge miskjendte deres Hjerters Tranq , hvortil Evangeliet vender sig . Og nåar ' de , for hvem Evangeliet vorder prcediket , bave en saadan Sindsforfatninq , kan Folqen ingen anden blive v. 23 , end at Lcerdommen om en korsfcestet Messias , Frelser , bliver for loderne mcd deres sanselige Messtasforventninqer en Forargelse , et Anstod , de hade den christeliqe Sandhed ; men for de hovmodige hedenske Visdomsqranstcrc en Daarlighed, at de nemliq skulde vente Frelse af en saadan Messias , som var et Anstod for deres Skjonhedssans , de foragte Christendommen ; saaledes ere loder og Hedninge tilsinds . Begge forsmaa Selvfornaqtelsens Vei , paa hvilken Man kun kan komme til at erkjende Evanqeliets quddommclige Sandhed som saadan ; Verdensstnd og Indbildssbed ere altsaa Hiirdrinqerne . — Men al Evangeliet alliqevel , uaqtet den Foraqt , som det <b>moder</b> hos saa Mange , er den lande <b>Visdom</b> , ser Man paa de Raldte v. 24 , som lade Naaden virle paa deres Hjerter og fore dem til Tro ; disse erfare det , og vistnok uden Forstjal baade loder og Grcekcre , at Cbristus , han selv , efterdi alle hine guddommelige Virkninger udgaa fra hans Person , er Guds Rraft ( Modsatning til lodernes Forargelse , ' hvilke ikke vilde tiltro en saadan Messias Noget ) , ved

<h4>3862</h4>1. V . 25. 26. De trcenctende Tidsomstcendigheder anbefale iscer det ugifte Liv . — V . 25. Sin Mening ( smlqn . v. 40. ) angaaende lomfruer, angaaende hvilke Cbristus under sttLareembede ftaa Jorden Intet har anordnet i bestemte Udsagn , heller ikke givet Apostelen nogen scerdeles Aabenbarinq, anbefaler Paulus med den Bemarkning : at Herren barmbjertiqen har forundt ham at vare tro , 3 : mit Raad fortjener Tiltro paa Grund af den Naade , som Gud bar stjcenket mig , til Trostab imod barn . Her og v. 40 adssiller Paulus Herrens Aabenbaringer noie fra det , som han ellers i Almindelighed erkjendte ved den Hellig-Aands Lys . — V . 26. Kun de trangselskulde Tidsomstcendiqheder og den endnu storre Nod ved Cbristi Gjenkomst anbefaler det uqifte Liv , hvilket Apostelen her tilraader , se 1 Tim . 4 , I — 3. 2. V . 27. 28. Doq ssal Man Me derfor oplose sit Wqtessab ; er du lost fra en <b>Hustru</b> , ugift , saa gift dig Me ; — v. 28 dog stal Man heller Me anse det for Synd under saadanne Omstcendigheder at gifte sig ; men de Gifte maa vente sig mere trykkende , lcqemliqe Tranqsler ; for disse vil den kjcerlige Apostel gjerne have dem staanet , dette er Grunden til hans Naad , ligesom ogsaa v. 32. 3. V . 29 — 31. Hvad den trykkende og tunge Tid fordrer af Enhver i enhver Stand . — V . 29. Forst Grunden til den folqende Formaning: Den endnu resterende Tid , inden store Tranqsler bryde los , er trang , er en trcengselsfuld Tid ; og sammes Hensigt er <b>Sjelens</b> Udlosninq fra Vcdhcenqenhed ved det Jordiske ; derftaa folqer nu selve Formaningen , som ssal vare Levnetsreqel for alle Christne : v. ' 29 De , som have Hustruer , ssulle Mc ved deres Kjarliqhcd til dem lade sig hindre fra at opofre sig for Troens Sag ; og ingen Sorq ssal saaledes overvcelde dem , som qrcede , at de tabc deres himmelske <b>Maal</b> af Siqte ; — v. 30 fremdeles stal ingen Glade og ingen Eiendom ; — v. 31 og ingen jordisk Nydelse fanqsse dem , bvilket vilde ste ved det lordisses Misbrug , fordi denne kommer cif syndig Kjarlighed til det Jordiske . — Hverken Besiddelse eller Tab ssal forstyrre den Christnes indre Fred og Sindsro . — End en Grund til saadan Selvforncegtelse og til Maadehold i alle Ting er den , at alt Jordisk er vaklende og utilforladeligt , og gaar en Forandring og Forherligelse imode 1 Joh . 2 , 8. Aabenb . 21 , ' 1.

<h4>4076</h4>» ) V . 29. Nogle Propheter stulle holde Foredrag , de andre Menighedslemmer stulle domme , om ogsaa Alt er overensstemmende med Skriften og Trocn 1 Thess . 5 , 19 — 21. smlqn . 1 Joh . 4 , 1 flq . b ) V . 30 — 33. Det andet Foredrag , af Den , som sidder der som Tilhsrer , og foler stg af Aanden dreven til et Foredrag , stal forst beqyndc , nåar det forste er endt v. 30 ; — saa kunne Alle efter Orden undervise , og modtage <b>Lardom</b> og Formaning v. 31. — Det er muligt at iaqttaqe den Fvennavnte Forskrift v. 30 , thi Propheterncs Vegeistrinq maatte ikke saaledes henrive dem , at de ikke vare sig selv mcegtige v. 32. Altid maa Sindiqbeden i den af den Hellig- <b>Aand</b> oplyste Fornuft saaledes beholde Overhaand , ' at Propheterne bleve bevande fra at indblande deres Eget i det Guddommelige og vidste at bedomme, hvad som just nu kunde vare heldbringende og velsignelsesrigt for Menigheden og i denne Forsamling ; Aandens Gave kommer frit ovenfra , griber og behersker den Troende , men denne behersker sin Tale , saa at han kan tie og , nåar han taler , veed han at finde det rette Udtryk . Rvgl . Nogle mene , fordi de have Forstand og den Hellig-Aands Gaver , at de ikke stulle vige for Nogen eller tie , deraf kommer Secter og Uenighet » ; men St . Paulus siger her , at de stulle og maa vel vige , efterdi ' Aandens Gaver staa i deres Magt , saa de ikke stulle bruge dem imod Enigheden , og ikte sige , at Aanden driver og tvinger dem . — V . 33. Og nodvendig er lagttagelsen af hin Bestemmelse , fordi Gud kun virker der , hvor Fred og Samdragtighed er , men Uorden vidner om noget Ugudeliqt , og det vilde vare Tilfaldet der , hvor Propheterne ikke rettede sig efter nogen Orden . 4. V . 34 — 36. Om Ovindernes Deltagelse i Gudstjenesten . — V . 34. Slutningsordene fta v. 33 foie sig bedre til v. 34 : Ligesom i alle Menigheder, saa stulle Eders osv. Ifolge Bestemmelsen i Loven 1 Mos . 3 , 16. smlgn . 1 Tim . 2 , 12. Eph . 5 , 22 stal Ovindens Underdanighed <b>vise</b> sig ogsaa deri , at hun tier i offentlig Menighedsforsamling , thi at tale undervisende udover et vist Herredomme over Tilhsrerne ; ogUndervisninq stal hun i Stilhed soge , ( nåar altsaa <b>Qvinder</b> talte Cap . 11 , 5. , saa var det imod den scedvanliqe christelige Skik , og Paulus forbyder det her udtrykkeligen ) v. 34 ; — V . 35. Paulus forbyder <b>Qvinderne</b> offentligen at sftorqe , fordi dette ofte maatte tjene til Paastud for upassende Tale . V . 36. Bevceggrund for Korinthernc til Lydighed ogsaa i dette Punct stal Apostelens Paaminbelse vare , at Christendommen jo Me er kommen fra dem , heller ikke ti ! dem alene ,

<h4>4784</h4>4. V . B — l 4. De Christnes Omvendelse og Oplysning skal for- nemmeligen tilbageholde dem fra hedensk Lastefuldhed . — V . 8. I vare Morkhed , helt iqjennem fuldte med Vildfarelse , Synd , Lastefuldhct , men efterdi I nu ved Eders Omvendelse fra Morket ere blevne frelste fra ten forriqe ulyksalige og uqudelige Tilstand , og ere blevne et Lys i Herren , en i Fonninqen med ham blevne oplyste og lyksaliggjorte , helt iqjennem fornyede , have og en et ganske andet Vesen , 1 Petr . 4 , 3. ' , saa omgaaes og nu som Lysets Born , forer Eders Levnet som oplyste og forbedrede Mennesker . - V . 9. Frugten , Virkningen , og Kjendeteqnet paa den i Hjertet virkcnte Hellig.Aand , er meddelende Godhcd , modsat Gjerriqbed ( v. 3 ) ; Retfcerdig : hed . helliqt Levnet , modsat Utuqt ( v. 3. ) og Sandhed i Ord og Gjerning , modsat skammelige Ord ( v. 4 ) og alt ban Skin og Hykleri ( v. 6 ) . — V . Il > . At omgaaes saaledes som Lysets Born er Eders hellige Pligt , desto mere . ta I ere istant » til , nttcliqcn at prove og tilforladellgenatindse , bvad HenenZ Vilje er og hvad som vinder hans Velbehag . — V . 11. Sclv meget langt fra al Forbindelse med de fordarvelige , ufrugtbare , til Intet tjenlige , kun Dodbringcnde Ondstabens og Ugudelighedcns Gjerninger , stulle II hellere ved grundede Forestillinger overbevise de Ugudelige om deres Gjerningers « Vtrafvardighed . — V . 12. Grund , hvorfor de Cbristne ingen Del stulle bave i et lastefuldt Levnet , er . at Hedningenes Laster , som ske lon- ligen af dem , en , som Morkets afstyelige Gjerninger Joh . 3 , 19 — 21 , saa skammelige , at den Christne nodigen endog kun taler om dem , fordi ban ilke kan det uden at rodme . — V . 13. Hvad Angaar den v. 11 givne Regel , at fremstille det Onde i sin Nogenhed , og at overbevise Andre om dets Forka < stelighed , saa oplyser Apostelen den ved en Sammenligning med Dagslyset . Dagslyset frembryder , udbnder sig over alle Gjenstande , og derved <b>vise</b> disse sig oplyste , i det klareste Lys . Ligedan er det med det Onde : Naar det overbevises ved Sandhedens Lys , bliver det aabenbaret , og nåar det saa er bleven aabenbaret , saa sttiar det der i sin Forkasteligheds klareste Lys . — V . 14. Hertil knyttes nu et lignende Sted hos Jes . 60 , 1. Samme » hang og Mening er : Ligesom det forholder sig med Oplysning ved Dags < lyset , saa er det og med den Ugudelige ; ban behover kun at vaagne op af sin aandelige Sovn , og staa op fra de Dode , fra sin Dod i Synden , saa vil Christus , som den rette Naadesol , fuldkommen oplyse ham . ( Ep . p. 20. Sondllg cft . Trimt . Cllp . 5 , 15 — 21 ) . 5. V . 15 — 20. Paulus vedbliver med mangehaande Formaninger . — V . 15. Han opmuntnr til en hoist forsiqtiq , besindiq Opforsel ; Vevceg- grund er den <b>Visdom</b> , som enhver oplyst Christen maa have . — V . 16. Met Klogstab ssal enhver Leilighed benyttes , som frembyder sig til at gjore tet Gode ; Rjober den rette , ' gunstige , passende Tid , Bevaggrund dertil er , at Tiden i det Hele er hoist farlig ^ ugunstig . Ndgl . Der <b>moder</b> den Christne faa mange Slags Hindringer og Aarsager , til at forsomme gavulige Forret- ninger , at ban maa ganste som en Fange slide sig los og ligesom stjale D < den ' , og endog dyrt nok betale den formedelst Ugunst , l , qesom Man siger : Venner ere Tidstyve . V . 17. De Christne ssulle beflitte sig paa Indsigt i ben guddommelige Vilje ; — v. 18 og Glceden i den Hellig.Aand Rom . 14 , 1 ? . , > Modsatning til Verdens syndige ' Lystighed , thi jo mere I fyldes af den Hc ! <

<h4>5467</h4>1. V . 3 — 5. Timotheus ' « farvede , oprigtige Tro foranlediger Apostelen til at takke Gnd . Sammenhangen er saadan : v. 3. Jeg takltt Gud , idet jeg uafladeliqen mindcs dig i alle mine Bonner , v. 5 takker ham for din Tro . Alt det Andre er udviklende Mellcmsatninqcr . — V . 3. Jeg takker Gud , hvem jeg tjener ligesom mine Forfadn , der haabede paa Messias , paa bvem jeg tror , fordi han er kommen ; i en ren Samvittigbed, i sand Gudsfryqt nemliq nu efter min Omvendelse til ham : for Apostelen , der foler stq sclv nar ved <b>Maalet</b> af sin jordiske Lobebane , Cap . 4 , 7. 8. , er Mindet om bans fuldkommede Forfadn dobbelt kjår ; — v. < denne Apostelens Erindrinq om Timotbeus kom af hans Lanqsel efter den elstede Discipel , hvilken bestandigen fornyede sig , saa ofte ban mindedcs te Taarer , som denne faldte ved Adstillelsen ; Taarer , Hjertets Dug , vidne om den dybeste Folelse , eller om det yderliqste Hykleri ; og Hensigten med Låna , , selen efter Timotbeus ' ftersonliqe Narvanlse var , at fyldes med Glcede over en saa dyrebar og ypperliq Discipel . — V . 5. Oprindelsen til demie opriqtiqe Tro bos Timotbeus er at soge i den qudfryqtiqe Opdraqelse , som bans troende Mormoder og <b>Moder</b> bave givet ham , og det ikke uden Virkning, tbi , siger Paulus , jeg er sikker paa . at den samme Tro bor ogsaa i diss . 2. V . 6. 7. Fordi nu Timotheus har en oprigtig Tro , knytter Paulus dertil fslnende Formaninger . — V . 6. Beliv , opflam , ligesom Man blaser en ulmende Gnist op til klar Flamme , ( isar ved Bon , ) i din stedse paany den dig af Gud ved din Embedsindvielse , bvilken stede untel Haandspaalceqqelse , smlqu . Avosthst . 8 , 15 — 17 : Cap . 19 , 6. ogsaa Cap . 13 , 3. 1 Tim . 4 , 14. , som Sindbillede paa den indre Meddelelse af Aanden fta Gud , forundte Naadegave , det freidige Sind til at qjore , vove , taale Alt for Guds ^ ag ; Timotbeus synes at van bleven modfalden og derfor mintic virksom , ivrig i Embedet ; — V . 7 Grund , tbi Aanden , som Gud skjenker Sine , er ikke en frvqtsom Forsaqtbeds <b>Aand</b> , som lader den forundte Naatc < gave übenyttet Matth . 25 , 26. smlgn . ogsaa Rom . 8. 15. , mcv meqet nmc en Araftens <b>Aand</b> , frimodigt og kjcekt Mods <b>Aand</b> , levende Rjcerligdcds <b>Aand</b> , og kloq « Vindighcd i al Gjerning ; Ndgl . Sindigt heder blandt oS at stikke sig ordentliqt , vakkert , fornuftigt . 3. V . 8. Skam dig heller ikke , af Fryqt for Spot og Forfolqclsc , vet Vidnesbyrdet . Forkyndclscn og deu offentliqe Brkjendclse til lesus Cbristus , eller ved mig , forn nu er i Fanqenssab for Christi Skyld ; men del og beer modigeu og taalmodiqt Forfolgelscrnc , som vederfares Evanqelict . 4. V . B — l 2. Tilskyndelse og Bevceggrunde til saadan Trostrc » kad under Lidelser . » ) V . 8. Lid ifolqe Guds Kraft - i : Gud sclv , den Almaqtiqc stjankcr Kraft dertil ; — v. 9 Han , som viser os de storste Velgjerninger , thi han bar gjort os salige , frelst os fra vor forriqe elendige Tilstand ; og bar kaltt os med et helliqt " af os antaqet Kald til Christendommen overhovedet og til vort Embede : Bevceggrund dertil var ikke egen Fortjeneste fra vor Tite

<h4>5579</h4>Vav . 2. Forskrifter for en <b>vis</b> og vcrrdig Embedsforsel af Titus . 1. V . I — s . Hvilke Formaninger Titus skal give de forskjcellige Menighedslemmer . — V . 1. Tal altid hvad som stemmer ovcrcus medden rene og sunde <b>Lcerdom</b> , desto mer , da hine Vranglarere afviqe derfra . Denne almindeliqe Regel anvendes nu paa de sarssilte Forbolde . — V . 2. Paabyd de gamle Mand sindig Opmarfsomhed paa sig selv sig . ; sindig , Lutber : tugtig . Rdql . overalt , hvorher staar sindig ( tugtig ) , der mark dig , at de ssulle vare fornuftige , forbolde sig ordentliqt og smukt , tbi jcq tor ikke bruqe Ordet fornuftigt . Ogsaa stulle de vcere sunde i Troen o : bolde sig med deres Tro alene til Guds sunde Ord . — V . 3. De bedaqede Ovinder stulle for . holde sig saaledes som det sommer sig for Hellige , Prastinder , helliqede til Gud , thi det ssulle Alle agte sig at vare , » : anbefal dem en Opforsel som er Christendommen vcerdig flq . ; — v. 4 at de med Ord og Gjeruinq undervise de unge <b>Qvinder</b> i deres Pliqter ; — v. 5 Hensigt , for at ikke Guds Ord , den cbristeliqe Låre , stal blive bespottet formedelst dens Bekjendens Laster . 2. V . 6 — B . Forman de ynqrc Mand v. 6 til Renhed i Sadcr ; — v. 7 men mer end ved Ord virke du paa dem derved , at du viser dig selv for Alle som et Forbillede i alt Godt , ved uforfalsket Lan , ved Nrbarbed » : ved en aqt < vardiq Opforsel ; — v. 8 og ved vclsiqnelsesrig og ulasteliq Tale , i den Hensigt, at ogsaa Ondstabsfulde maa skamme sig over deres fiendske Sindelaq , og intet Ufordelaqtiqt kunne sige om os . 3. V . 9. 10. ' Forman Tjenerne til Lydiqbed , at de ikke modsiqe 3 : at de adlyde uden Indvending ; — v. 10 og til Trostab , for at ogsaa de i enhver Henseende maa qjore ben christeliqe <b>Lcerdom</b> LLre , nåar det ucmliq erkjendes , at denne Låre danner saadanne gode , troe og nuttiqe Tjenere . 4. V . 11 — 14. Bevceggrunde , som forpligte og tilskynde Enhver til en hellig Vandel , hentede fra Guds Naade , der saa herligen har aabenbaret sig i Forlosningsvcerket ved Jesus Christus . : > ) V . 11. Den for kun lovede saliggjørende o : Syndere frelsende Guds Naade er aabenbaret , er , liq Solen , opqanqen mcd sin Glands oq Her . liqhed , oq dette i storste Almindeliqbed over eller for alle Mennesker , Mrc . 16 , 15. I dcnne vcd Jesu Christi Sendelse aabenbarede Guds Naade ligger Grunden til al Frelse og Bevceggrundcn til vor Helliggjorelse . b ) V . 12. Den ncermeste Henstgt med denne Guds Naade , som oplcerer

<h4>6241</h4>den storste Overensstemmelse Sted , bvad Indhold , Fremstilling og Sprog anqaar ; der som her hersker den dybeste , iuderliqste Fslclses Sproq , en liqesaa dyb som simpel Fremstilling og en hoist utvunqen , naturlig Rakke af Tanter , fremstillcde i ssjon Orden . Angaaende Liden , nåar det er skrevet , angiver Oldtiden intet Bestemt , og Brevet selv tier saa aldeles derom , at Man dcsanqaaende bar nasten kun Formodninqer; doq er det hoist sandsynliqt , at Brevet er bleven skrevet efter Evangeliet,'paa hvis Indhold dets Beqyndelsc umiskjendeliqcn taqer Hensyn ; derfor bave Mange troet , hvad doq ikke er bcvisliqt , at Nrevet oprindeliqen maa have vceret en Folqestrivelse til Evanqcliet ; i ethvert Fald ere begge To meget yngre , end selv de ynqstc Breve fta den helliqe Paulus . Liqesaa übckjcudt som Tiden er ogsaa Stedet , hvor Brevet er strevet ; maasse er det forfattet i Epbesus , men ialfald i Lilleasien , hvor Apostelen virkede i stn hoicre Alrerdom . Modtagerne vare hoist sandsynliq hine Menigheder , til hvilke Brevene i Johannes ' Aabenbaring Cap . 2 , 3. ere stilede , thi den i Vrevet herskende Tone er en Larers og Vens , som er Nne Lasere bekjendt og bengiven , og bine Meuiqheder i Forasicn borte til den Virkekreds , i hvilken Apostelen arbeidede . En gammel Overskrift til Brevet siger vel , at det er skrevet til Partherne , men det er en aldeles übevisliq Efterretninq , at Johannes bar pradiket Evangelium for dette Folk ; og da Brevets Modtaqere , ( det er tydeliqen en Omganqsstrivelse til flere Meniqheder , ) advares for hedenske Laster og saadanne vildfarende Lardomme , som allerede i Pauli Breve tilKolossaerne og Eftbeserne beteqnes : saa er det mere end sandsynliqt , at Modtaqerne vare Hedningechristne , som levede i den ftemmerste Del af Lilleasien , hvor en saadan Aands-Netning var udbredt . Anledningen til og Hensigten med Brevet eranfort Cap . 5 , 13 ; Johannes vilde formane sine Lasere , ' Hcdninqcchristne , til Standbaftighed og Trostab i den christelige Tro , vilde qjore dem opmarksomme paa Hovedmanqlerne ved dereS Cbristcndom, vilde fornemmeliq indstjarpe dem Helliggjorelsens og Broderkjcerlighedens Pligt , og derved sikre dem for alle AntichristcnS Forforelser . Da Brevet har denne Hensiqt , saa er Vranqlareres V ckjampclse og VranqlareS Gjendrive ! se inqenluudc udeladt , men kun et underordnet og middelbart Oicmed for Apostelen, som ikke bedre kunde sikre og advare Brevets Modtaqere for alt Uchristeliqt i Låre og Liv , end nåar han ret alvorliqen og varmt lagde dem den christelige Sandbcd selv og de deraf flydende Pliqter paa Hjertet . Den belliqelobannes ' Advarsler ere i ' Almindelighed rettede derben , at de Cbristne alvorliqen oqomhyqqeliqen ssulle voqte sig for Verden , for alle uqudeliqt sindede Mennesser, oqfordc Forforelser , som fremkom af deres Omqivclsers vcrdsliqe Diqten og Tragten . ' I Scerdeleshed bekjamper ban saadanne antichristcliqe Vildfarelser og Forforelser , som truede de Troende inden for det cbristcliqe Samfund . En falsk <b>Visdom</b> , en forcqiven dybcre Indsiqt gjorde sig allerede denqanq qjceldcndc , og idet mange Vranglarere dels aldeles nceqtede det Historiske i Christendommen , dels beskrev det som noget aldeles Betydninqslost , saa forfalskede de den sande Cbristendom og ftemkaldte Tvivl om Sandheden i Forloserens mcnncsseliqe Aabenbarelse og Historie . For saadanne Forforere , der tildels vare frafaldnc Cbristne Cap . 2 , 19. , og som naglede Christi sande mennestelige Natur , advarer Johannes udtrykkeliqen Cap . 4 , 2. 3 ; men ogsaa , og isar , for en Christendom , ved hvilken Man ikke beflittede sig paa Helliqbed , og derfor formaner ban i det bele Brev saa mangestdiqen til et christcliqt Levnet , som maa vare Bevis og Kjendeteqn paa den aqte Tro . Med Hensyn hertil siger Luther i sin Fortale til dette forste Brev : „ St . lohannis forste Epistel er en retstaffen apostolist Epistel , og ssulde billigen folge „ strar efter bans Evangelium . Thi ligesom han i Evanqcliet driver ftaa Troen , „ saaledes <b>moder</b> han i Epistelen , dem , som rose sig af Troen uden Gjerninqer . Og „ lcrrer manqcsidiqen , at Gjerninqerne ikke udeblive , nåar Troen er der . Men ude„blive de , saa er Troen ikke retstaffen , men Logn og Morke . Men dette gjor ban „ ikke ved at drive paa Loven , ligesom St . Jacobi Epistel gjor ; men med Ande , „ forat ogsaa vi ssulle elsse , ligesom Gud bar elsket os . Men han skriver ogsaa deri „ haardeligen imod Antichristens <b>Aand</b> , som allerede denqanq begyndte at naqte „ Christus , at han er kommen i Kjodet , bvilket nu forst ret gaar i Svang . Thi „ om Man end ikke nu offentlige » ! med Munden nagter , at Cbristus er kommen i „ Kjodet , saa naqte de det med Hjertet , med Låre oqLevnet . Thi hvo , som vil blive „ retfardiss og salig ved sine Gjerninqer og sin Virken , Den gjor just sacimcget som Den , „ der forncegter Christus : efterdi Christus derfor er kommen i Kjodet , foråt han „ kunde qjore os retfardiqe og salige uden vore Gjerninqer , alene ved sit Vlod . Folkelig bekjcemper Brevet begge Parter , baade dem , som ville vare i Troen aldeles „ uden Gjerninger , og dem , som ville vare retfardige ved Gjerninger . Og hol„der

<h4>6768</h4>alle Mennesker , som cre Tilhanqere af Dyret , Indvaancre af Staden , Tilbedere af <b>Qvinden</b> , over Niqcts ' forrige Magt og Fald , bvoraf det doq igjen reiser sig , saa at det imod Forventnlnq staar der igjen med ny Krast . (Liqcsom Gud og Christus beteqnes som den Evige , der var og er og stal vcere , som den Trofaste , Tilforladeliqe og Eviqvarcndc , saa blivcr ber stik i Modsatninq Dyrets Nige , den christne Kirkes Modstander , beteqnet som et Skinvesen , som var og ikke er , kommer og farer hen ; dets bele Tilvarelse er en giftig , burtigt sondersprinqrnde Vandblcere , en fiendtliq Damp , en forqjanqeliq Helvedestaage ; tbi alt Ondt og Uqudeliat har ingen sand og blivende Tilvarelse . ) — " V . 9. Her bchoves et Sind , som haver ) ) isdom , 0 : her <b>vise</b> siq Indsiqten , om Noqen bar <b>Visdom</b> , liqesom der Cap . 13 , 18 bliver henvist til det fornodne Skarpsind , for at forståa disse qaadcfuldc Ords stjulte Menjnq .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111303008'target=_>Nohrborg, Anders, 1859, Det faldne Menneskes Saliggjørelses-Orden</a><h4>6025</h4>ligen ud paa at kunne have Ro , og at komme tilbage til sin Ro , naar denne forstyrres og han bliver urolig . Et Menneskes Ro her i Verden har mangehaande Hindringer og Modstand ; thi at Verden for Syndens Skyld er en Jammerdal , at Jammer og Elendighed <b>moder</b> os ved vor fsrste Indiredelse i Verden , fslger os under hele vor Vandring gjennem Verden , og forlader os ikke , fsrend vi , om det steer i en salig Stund , faae gaae ud af Verden, og vort trette Legeme lceggcs ned i Jorden, hvoraf det er sammensat , og fores ind i fit Hvilckammer , det er en Sandhed , som Erfaringen dagligen lerer os . Og i Serdeleshed faae vi erfare , at den naturlige Ufuldkommenhed i alle Ting , som omgive os ; at de Menneskers Kjerlighedslsshed , egennyttige Hensigter og Gjerninger , blandt hvilke vi boe ; at Tiderncs Fordervelse , i hvilke vi leve ; at vort eget Legemes Skrsbelighcd , som fslger os , vore Lysters og Begjeringers Uorden , som staae omkring og henge ved os , og flere Ting , som hertil hore , ere alle Fiender og Hindringer for vor <b>Sjels</b> Hvile og Ro , og foraarsage , skjsndt i ulige <b>Maal</b> hos den Ene og den Anden , at stor Urolighed er stavt for hvert Menneste. og der er et tungt Aag paa Adams Born , fra den Dag , de udkomme af deres Moders Liv , indtil den Dag , de komme til Alles <b>Moder</b> igjen- ) .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009082403015'target=_>Linderot, Lars, 1850, Lars Linderots Prædikener paa alle Søn- og Helligdage</a><h4>1171</h4>denne hedenske <b>Kvinde</b> vover Alt ! — Men hvad <b>moder</b> hende nu ? Ak , det andet Slag rammer hende paa samme Sted . Da Jesus sik se hende i Huset , saa gik han ud . Da lob hun efter ham paa Veien , og raaber , men han svarer hende ikke . Ak , nu maa jeg sporge : hvordan er det , Herre Jesu ! Vil du ikke svare ? Du evige Guds Faders Ord , vil du ikke tale ? Du evige Guds <b>Visdom</b> , kan du ikke svare ? Du kyndige og store Lege , veed du ingen Udvei ? Du evige Barmhjertighet ) , lider du ikke ved at se et Menneskes Elendighed ? Du Naadens Brond og levende Vandkilde, sprudler du da ikke lengere . Er du hentorret ? Er din Mistundhed ude evindelig , og har dine Tilsagn faaet Ende fra Slegt til Slegt ? Har Gud glemt at vere naadig , eller har han tillukt sine Barmhjertigheder ved Vrede ( Ps . 77 , 9. 10 ) ? Barmhjertige Herre Jesu ! Naar pleiede du ikke at svare en betrengt <b>Sjel</b> , som raaber til dig ? Syge og Bedrovede forfulgte dig i lodeland fra Morgen til Aften med Raab om Hjelp , og altid var du naadig , altid svarede du . Men hvad lese vi nu ? Der kommer en hedensk <b>Kvinde</b> til dig , og du svarer hende ikke et Ord . Ak , stal hun vere den eneste i Verden , som du ikke vil forbarme dig over ? Endda lader hun Modet ikke falde , endda vover hun Alt , endstjont Proven bliver hende sverere at beståa . Jesu store Barmhjertighet ) holder sig tilbage mod <b>Kvinden</b> ( Es . 63 , 15 ) . Det lader til , at han var übevegelig for hendes Bon ; ja det synes som Disiplene vare folsommere end han . Da hun merkede , at hun ikke fik Svar af Frelseren selv , ikke et Ord engang , hverken ja eller nei , saa gik hun sikkerligen hen til Disiplene , og bad dem at tale hendes Sag , „ I have den Naade at vere optagne i denne store Mesters Skole , " har hun vist sagt . — „ I staa i en serdeles Naade og Gunst hos ham . Efterdi det ikke nytter , at jeg raaber , saa tal I til ham for mig , at han hjelper mit plagede Barn . Ak , kjere Herrer , jeg beder Eder saa hjerteligen , gjorer mig , arme <b>Kvinde</b> , denne Tjeneste . " Og i denne Bon maa hun have veret saa indstendig , at Disiplene paa ingen Maade kunde blive hende kvit . De blive rorte . De trede derfor frem til lesum med deres Forbon . De sige : Ski.l dig dog af med hende , thi hun raaber efter os ! Ingen maa tenke , at de rålede saaledes af Ufolsomhed , som vare de ligegyldige , hvad

<h4>1191</h4>Men hvad som er strevet om hiin <b>Kvinde</b> , er strevet os til <b>Lerdom</b> ( Rom . 15 , 4 ) . Nu vil jeg atter tale til dig , du under din Sogen efter Naade og i din Naadestand haardt anfegtede <b>Sjel</b> . Blandt alle Fristelser , der anfegte et bcdrovet Hjerte , er det en saare sterk Fristelse , nåar Leren om det evige Naadevalg anfegter den Bedrovede . Ak , hvo bliver ikke ncdboiet , nåar disse Tanker , ligesom en Flod , oversvsmme <b>Sjelen</b> , og ville drukne alt Haab om Naade , disse engstende Tanker : Jesus er ikke udsendt til at frelse dig . Han er ikke udsendt uden til saadanne Syndere , som

<h4>1194</h4>ikke have forsvet saa grove Synder , som du har gjort . Guds Sindelag mod Menneskene er forstjelligt . Imod Nogle er han mild , mod Andre streng . Nogle har han besluttet i Evighed at gjore salige , og Andre har han foresat sig at fordomme . Var du saa heldig , som Petrus eller Paulus , saa kunde du vente , at Gud stulde hore og hjelpe dig . Men du er jo ikke udvalgt . Du var ikke tilstede ved det evige Naadevalg . Du er ikke med i de Udvalgtes Tal . Af alle Pile , der ramme Hjertet , er nesten ingen saa starv , som Kvaler over Udvelgelsen . Men stulde her vere en <b>Sjel</b> , der er <b>Maal</b> for disse starve Skud , saa bliv dog ikke tret i dit Sind , du forferdede <b>Sjel</b> ; kast ikke bort din Forhaabning om Saligheden , men vov Alt , og du stal vinde . Det gaaer ingenlunde an , at du lengere tretter med Gud eller kjemper med din blinde Fornuft . Det gaaer ikke an at trede op i Guds Raadkammer , og soge at udforste den Eviges Raad ; det er dig heller ikke befalet ; men bliv du hernede . Den kanaanitiste <b>Kvindes</b> Raad er det allerbedste . Hvad gjorde hun ? Jo , med Bonnen trengte hun sig frem . Da Jesus sagde : Jeg er ikke udsendt til dig , saa kom hun krybende til ham paa Veien : han fik ikke gaae , thi hun holdt fast i ham , og sagde : Herre hjelp mig ! Du , om din Udvelgelse mistvivlende <b>Sjel</b> , gjor ligesaa ! Naar det i Fristelsens Time hvisker i dit Hjerte : Jesus er ikke sendt til dig , han angaaer dig ikke , saa kom i dybeste Vdmyghed krybende frem til Jesu blodige Kors , og sig : Herre , hjelp mig ! Jeg holder paa at drukne i Naadevalgets Hav ; Herre , hjelp mig ! At hjelpe er dit Embede , dit store Kald , derfor er du kommen i Verden , Herre , hjelp mig ! Hvorledes det henger sammen med Udvelgelsen , ved jeg ikke , men at du er den hele Verdens Hjelper , det veed jeg , thi det er dit Navn ; Herre, hjelp mig ! Jeg vover ikke lengere atfordybemig iden vanskelige Undersogelse , om jeg er udvalgt eller ikke . Tilbage til Naadevalget , som stede i Evigheden , tor jeg ikke gaae : nei , jeg beder i Tiden , jeg raaber kun om Hjelp i Jesu Navn ; Herre hjelp mig ! See , den bedste Maade , hvorpaa man kan blive kvit denne Fristelse , er ved Bonnen at paa- . minde Frelseren om hans Embede . Da griber du ham ikke blot i hans Kledebon , som den blodsottige <b>Kvinde</b> gjorde :

<h4>3704</h4>En Kristen mistvivler aldrig , nåar han engang har erfaret Guds Naade og Hjelp i sin Bedrovelse . Med Taalmodighed oppebier han Tiden , thi han veed , at han Me har forglemt ham , og at hans Venten er til hans eget Vel . Under saadanne Forvold kan jeg idag , elstede Tilhorere og Venner , ikke onste Eder noget bedre , end hvad Paulus onstede Filippenserne : maa Eders Kjerlighed fremdeles blive mere og mere overflodig , udi Erkjendelse og allehaande Fornemmelser , at I kan prove , hvad bedst er ( Filipp . 1 , 9 ) . Det er i Sandhed en stor Klogskab , nåar man forstaaer at anvende sine Erfaringer til sin Nytte , saa at man herefter handler forsiktigere mod Menneskene og setter storre Tillid til Gud . Men det er ogsaa meget vel , ja hoist nodvendigt , at besidde den aandelige Erfaring om Guds skjulte Veie . Dette er vor sande Sjelestat , og Gud kan ved at tildele os sin Naadegave Me have tåget Hensyn blot til vort legemlige , men og til vort aandelige Vel . Ak , hvor lyksaligt er ikke det Menneste , som i sin <b>Sjel</b> alt har fornummet Guds Gjerninger , og Me blot udenad lest , men og virkeligen erfaret , hvad Bod , Tro , Retferdiggjorelse , Helliggjorelse , Guds Kjerlighed , hjertelig Bon , <b>Visdom</b> , Naadestanden og den evige Salighed har at betyde . De , som ere rige paa anden Kundstab og Erfaring , kunne ogsaa derefter prove , hvad der er bedst , prove , hvad der er Guds Villie , og berede deres Hjerter til med samme Beredvillighed at bere saavel Sorg som Glede . De vide , at hvad som steer , hvad som <b>moder</b> dem , er til deres Bedste og deres Forbedring. Hvor forherdede og uhelbredelige er Me de , som Me lade sig omvende ved Ulykker og Gjenvordigheder , men forherde sig mere og mere , eftersom den Eviges Haand

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012102924011'target=_>Holberg, Ludvig, 1843, danske Skueplads, eller Ludvig Holbergs samtlige Comoedier</a><h4>3543</h4>peer . I Europia . Jens . Ja , det er troligt nok ; jeg kan hore paa dit <b>Maal</b> , at du est en Europcrer , og ingen Polakker ; men paa hvad udi Landet est du fod ? peer . Udi Kiobenhavn . Jens . I hvad Gade udi Kiobenhavn ? peer . I Klceboerne . Jens . Ja , saa sial du lade dig kalde Peter eller Peiter von Klccboe . peer . Jeg faaer vift Skam , jeg staner her saa , jeg har saa meget at forrette for min Herre . Han vil giore Vers til en <b>Pige</b> paa Fransi , som han er forlibt udi ; men som han ikke er studeret selv , har han bedet mig at oplede en Poet , som i Hast kan smede noget sammen for ham . Jens . Jeg sial <b>vise</b> dig En , som gier Vers for en billig Priis . peer . Hvor boer han ? Jens . Han boer udi Aabenraae , ncrst ved Guldsmeden . peer . Hvad er hans Navn ? Jens . Jeg troer ikke , han har noget Navn ; jeg har aldrig hort , at han kaldes anderledes end Poeten i Aabenraae . peer . Jeg veed jo . Poeter blive dobte som andre Mennesker ? Jens . Det sial jeg ikke egentlig kunne sige dig ; nok er det , at du sporger efter Poeten i Aabenraae . peer . Tak for god Underretning . Jeg vil gaae lidt ind til Christoffers , og drikke en Potte M forst , siden vil jeg hen at lede den gode Poet op. Adios . Ga ^ er . , Jens . Serviteur . Hvem Pokker mon det er , han er bleven forlibt udi ? det er en gangbar Syge i disse Dage . Mig synes , min Herre er heller ikte rigtig . Gaaer üb .

<h4>3580</h4>Det cr et underligt Sfags Folt , disse Poeter . Om en anden cerlig Mand giver et Oresigen udi Tanker , trccfker man ham til Daarekisten ; men hos dem er Saadant Tegn til <b>Lcerdom</b> og Dygtighed . Ellers sit jeg det Vers for godt Kiob . De Folt maae have nogen Frihed , efterdi de ere saa lidet interesserede ; thi hvad er dog tolv Skilling for saadan lcrrd Mand ? Jeg forstaaer ikke Verset , men kan dog see , at det er altid sine Penge vccrd . Nu skal jeg bilde min Husbond ind , at jeg har givet to Rirdalcr derfor , saa viuder jeg ved dette Mrende elleve Mark og sire Skilling . Jeg har kuu liden <b>vis</b> Len , maa derfor ncere mig redelig ved Commissioner . Mangen brav Mand foder <b>Kone</b> og Born derved udi Kiobenhavn ; thi hvoraf skulde de gode Folk ellers leve ? Hvio jeg kunde flattere , som Jesper gier , for min Herre , tunde jeg have de Smauser og de Forceringer , som han faaer . Jeg skulde ilte rose mig selv ; men jeg cr en cerlig Karl , der ikke kan sige andet , end jeg mcncr . Naar min Husbond begynder at tale om sin Manddom i den Vrabantsie Krig , tier Peer ganske still ; thi jeg veed , ban har aldrig vceret paa den anden Side af Velt , end sige udi Brabant , som ligger nogle tusinde Mile herfra . Nei , Wrlighed varer dog lcrngst ; det hedder hos mig , som Hollænderne siger : Thue das Recht nicht , und siv den Teuffel . Kunde jeg saa vel Tydst , som jeg kan Hollandsk , var Peer anden Mand , cno ban er ; thi jeg haver tidt heri Alerander Skolemester sige : hvo der kan Tydsi , cg haver nogle Midler dertil , kan reise hele ( s ' uropia igienncm . Men Fanden er udi dette Dn , Das , det kan man allene fast i fiorten Dage ikke lcere . Gaaer bort .

<h4>10489</h4>petronius . Der seer jeg jo en ung Philosophus , som taler med min Herre . Den Karl maa tidlig vcere bleven Doctor . Pernille . Lad Eders Famulus gaae til Sive , medens vi tale sammen . petronius . Hvis det ikke var en Doctor, saa vilde jeg svcerge paa , at det var Leonoras Pernille ! thi de ere hinanden saa lige , som de kuude vcere stoble i een Form . isosmoligoreus . Hor , Petroni ! gak til Side , saa lcrnge jeg taler med denne Herre . perronius . Jo meer jeg beskuer ham , jo meer han ligner hende . Pernille . Lad ham gaae reent bort ; thi jeg har andet af Vigtighed at tale om . Gak tunhiem til dine Forretninger , Petroni , til jeg kalder vaa dig . petronius . Ach Herre ! jeg har og Lyst at tale med denne Fremmede ; han ligner en af mine Kiendinger . Pernille , Gaa bort , naar Jer Herre befaler Jer . Sagte . Gid du faae en Ulykke med dine Kiendinger ! hoit . Hvad har Saul at giore blandt Propheter ? Gak strar bort , som jeg siger . ( luil ! 8 « ul inter l ^ - aplietas ? petronius sagte . De sige : hvad har Saul at giore blandt Propheter ? Jeg kunde jo svare med en <b>vis</b> <b>Dame</b> , han ledte efter sin Herres ( s ' sler . Ha , ha , ha ! <b>Maalet</b> er ogsaa , gid jeg faae Skam ! Pernilles . Ha , ha , ha . Gaacr bo « .

<h4>10503</h4>Jeronimus . Ha , ha , ha ! det er en forslagen <b>Pige</b> . Jeg mcrrler , at hun vil have hans skriftlige Anscgning i Hcrndcr . for at overbevise hendes Frue , at han er kun en Philosophus af Navn . Leander . Og at han er en Hykler og Menssalk . Jeronimus . Det dog ikke saa meget , som for at <b>vise</b> , at han er allene udi egen Indbildning Philosophus . thi det kan vcrre , at han selv bilder sig ind at vcrre det , han virkeligen ilte er ; ligesom man seer mange , der udi philosophisie Skoler og Collegier oplccres udi Vildfarelser , og faaer en falsk Idce om Philosophie , saa de bilde sig ind , at den bestaaer udi Kaabe , Ekiceg , Egensindighed , eller udi puur Theorie ; skiondt de rette Lcrremestere bor ikke saa meget beflitte sig paa at giere deres Disciple lcrrde , som at giore dem gode og dydige . Nu er han kommen nnder anden Information , som er end meer gal , end den , han har faaet i sin Skolegang . Leander . Monsieur Jeronimus mener den pragifie Philosophie ? Ha , ha . ha ! Saa tidt jeg tcrnker derpaa , maa jeg forundre mig over denne <b>Piges</b> Capaeitet . Jeg var allene bange for , at hun outrerede Tingen for meget . Jeronimus . Jeg ikke ; thi naar man ftrst har indprentet Nogen hoie Tanker om sin Person og <b>Lcrrdom</b> , kan man siven let faae ham til at troe , hvad man vil , helst naar det flatterer Ens egen Tilboielighcd . Thi ingen <b>Lcerdom</b> kunde komme Cosmoligorco meer til Pas , end den , som indeholdes udi det pragisic Morale , hvorudi Rangsyge giores til en honnet Ambition og til en philosophisi Dyd . Leander . Meu hvad sial man sige om > den fiortende Artikel , som tillader en Mand at ligge hos sin <b>Pige</b> to Ncrtter om Ugen , : ! uden at bedrive Synd ?

<h4>11491</h4>gaaet ham i at reformere Comoedier som den politisie Kandestøber i at reformere Staten, hvis Crempel jeg haaber han ogsaa udi Catastropben efterfslger , ved at opbrande de Skrifter , hvis Låsning har forledet ham til saadant , ligesom den Politiske Kandestober giorde med sine politisie Nachtischer eller Stoksisie , der bragte ham i de Griller at Hamborg maatte nsdvendig gaae i Grunde , hvis han ikke blev Borge- render , ! min Skrivemaade , og at jeg har et lidet Pund at komme « < " l siaae mig les ; her gives mig den rigeste Materie , jeg min Livs-Tid har havt til , aadant . engang , hvor det vilde gaae Eder , om jeg vilde giore NepreB « » , iei ' om jeg vilde , som jeg efter saadan haanlig Provocation har Ret til , eramlnere hans Comoedier igien ; thi om jeg vilde giore det endoqsaa med den som han kalder ! it allerbedste Stykke , Den Eeendis Blinde , siulde han selv forundre stq over hvordan han var kommen paa de Tanker at agere Comoedie-Skriver . leq vilde susi / at ieg aldrig havde brugt nogen Neiircssi > lier mod nogen af dem , der har angrebet mig tilforn ; vil herefter strabe at overvinde den Passion , og derfor ikke hevne mig ved » aadant foretagende , men allene denne Gang , for at fore ham af Vildfarelse . Vile , af nogle Poster , hvor ulykkelig han har varet i at omstobe dette Stykke . Den iste Stobning er udi Begyndelsen , hvor her gisres et Uvsteron prnteion , og det lom for var Fortale , bliver giort til Bagtale , efter hvilken Regel i crllwuen kan ikke vides . Vist nok er det , at Hlonsiem- har ikke raadfort sig med som gav Regler i gamle Dage , ei heller d ' ^ uliissiinL , som er Velviser i vore Tider Med c « ! ! eFiu politi . : « , til hvis <b>Kundskab</b> jeg efter Reglerne har banet Vei i den hele fyrste Aet , begynder Aonsieur strar sin omstobte Comoedie , og derved synes at imitere det , , om de Hollandske Linie-Dandsere kalder i deres Nacl ' spil hvor Tilfluerne blive forvirret ved at sce eller hore noget , som de ingenlunde kan begribe , hvad det sial vcrre , forend de har seet Enden ; eller visse tydsie Romaner , der begynde undertiden saaledcs : " Gewisi , erwiedcrte der ungluckselige Ritter Philander , " hvis Betydelse man oste ikke faaer at vide , forend man kommer noget hen udi den tolvte Bog af Romanen ; da dog en Hoved-Regel udi Comoedier er , ikke at forestille noget uden man tilforn har givet Tilskuerne en tilstrækkelig Kunosiab derom ; hvilket jeg noie har iagttaget r denne og alle mine andre Comoedier . Dette store « vsternn p. ateron hedder hos ham Monsieur , en Reformation , som dog er tilstrækkelig uden Hielp at fordcerue et Stykke allene . Saaledcs : Det ferste Dattet trattes op , sees en Hoben Folk siddende ved et Bord . og Gn begynder Comocdien med disse Ord : " Det vil give sig Paa ncrste Rigs- Dag , " uden at man veed , hvad det er for Folk , og hvad de vil sige . Den 2 den Stobning oestaaer derudi , at von Bremens <b>Kone</b> . Gedsie , bliver giorl til hans Svigermoder , det er , af en myndig <b>Kone</b> , som har Part i hans ringe Stand , aavel , om hans Hoihed , og derudover med fast storre Virkning spiller sin Rolle udi denne Comoedie , end Manden selv , bliver en Svigerste , som hverken blir fattig eller riig . hverken hoi eller nedrig ved hans Forandring , ' og som han derforuden kunde efter de fleste Sviger , onners Erempel <b>vise</b> Dorren , naar ham lystede . Og derved er den anden Hoved-Person i Comoedicn totnlitei- ruineret , og det , som var mest naturligt cg anseeligt , giort tvungen.og unaturligt : imidlertid maa dette dog ogsaa hedde en Reformation.

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012102924022'target=_>Holberg, Ludvig, 1852, danske Skueplads, eller Ludvig Holbergs samtlige Comoedier</a><h4>3368</h4>peer . I Europia . Jens . Ja , det er troligt nok ; jeg kan hore paa dit <b>Maal</b> , at du est en Europcecr . og ingen Polakker ; men paa hvad Sted udi Landet cst du fed ? peer . Udi Kisbenhavn . Jens . I hvad Gade udi Kisbenhavn ? peer . I Klceboderne . Jens . Ja , saa stal du lade dig kalde Peter eller Peiter von Klceboe . peer . Jeg faaer vist Skam , jeg staaer her saa , jeg har saa meget at forrette for min Herre . Han vil gisre Vers til cu <b>Pige</b> paa Fransk , som han er forlibt udi ; men som han ikke er studeret selv , har han bedet mig at oplede en Poet , som i Hast kan smede noget sammen for ham . Jens . Jeg stal <b>vise</b> dig En , som gisr Vers for en billig Priis . peer . Hvor boer han ? Jens . Han boer udi Aabenraae , ncest ved Guldsmeden . peer . Hvad er hans Navn ? Jens . Jeg troer ikke , han har noget Navn ; jeg har aldrig hort , at han kaldes anderledes end Poeten i Aabenraae . peer . Jeg vccd jo . Poeter bliver dsbtc som andre Mennesker ? Jens . Det stal jeg ikke egentlig kunne sige dig ; nok er det , at du spsrger efter Poeten i Aabenraae . peer . Tak for god Underretning . Jeg vil gaae lidt ind til Christoffers , og drikke en Potte Ol fsrst , siden vil jeg hen at lede den gode Poet op. Adiss . Gaaer . Jens Servitcur . Hvem Pokker mon det er , han er bleven forlibt udi ? det er en gangbar Syge i disse Dage , Mig synes , min Herre er heller ikke rigtig . Gaaer ud .

<h4>3406</h4>Det er underligt Slags Folk disse Poeter . Om en anden cerlig Mand giver et Oresigen udi Tanker , tratter man ham til Daare- Kisten ; men hos dem er saadant Tegn til <b>Lcrrdom</b> og Dygtighed . Ellers sit jeg det Vers for godt Kiob . De Folk maae have nogen Frihed , efterdi de ere saa lidet interesserede; thi hvad er dog tolv Skilling for saadan laud Mand ? Jeg forstaacr ikke Verset, men kan dog see , at det er altid sine Penge vcerd . Nu stal jeg bilde min Husbond ind , at jeg har givet to Rirdalcr der » for , saa vinder jeg ved dette Wrende elleve Mark og fire Skilling . Jeg haver kun lidm <b>vis</b> Lon , maa derfor ncere mig redelig ved Commissioner . Mangen brav Mand soder <b>Kone</b> og Bom derved udi Kiohenhavn ; thi hvoraf skulde de gode Folk ellers leve ? Hvis jeg kunde flattere , som Jesper gior , for min Herre , kunde jeg have de Smauser og de Forceringer , som han faaer . Jeg skulde ittc rose mig selv ; men jeg er en cerlig Karl , der ikke kan sige andet , end jeg mener . Naar min Husbond begynder at tale om sin Manddom i den Brabandste Krig , tier Peer ganste still ; thi jeg veed , han har aldrig vcerct paa den anden Side af Bclt , end sige udi Brabant , som ligger nogle tusinde Mile herfra . Nei , Wrlighed varer dog lcrngst ; det hedder hos mig , som Hollcendcrnc siger : Thue das Recht nicht , und sty den Teuffel . Kunde jeg saa vel Tvdst , som jeg kan Hollandsk , var Peer anden Mand , end han er ; thi jeg haver tidt hort Alexander Skolemester sige : hvo der kan Tydst , og haver nogle Midler dertil , kan reise hele Europia igiennem . Men Fanden er udi dette Der , Du , Das , det kan man allene fast i fior-

<h4>10329</h4>petronins . Der seer jeg jo en ung Philosophus . som taler med min Herre . Den Karl maa tidlig vorre bleven Doctor . Pernille . Lad Eders Famulus gaac til Side , medens vi tale sammen , perronius . Hvis det ikte var en Doctor . saa vilde jcg svcrrgc paa , at det var Leonorcs Pernille ! thi de cre hinanden saa lige , som de kunde vcere stsbtc i ecn Form . < sosmoligorens . Hor , Pctroni ! gak til Side , saa lcengc jeg taler med denne Herre . petronius Io mecr jeg beskuer ham , jo mecr han ligner hende . Pernille . Lad ham gaae rcent bort ; thi jeg har andet af Vigtighed at tale om . Cosmoligorens . Gak kun hiem til dine Forretninger , Petroni , til jeg kalder paa dig . petronius . Ach Herre ! jeg har og Lyst at tale med denne Fremmede ; han ligner en af mine Kicndingcr . Pernille . Gaa bort , naar Jer Herre befaler Jer . Sagte . Gid du faac en Ulykke med dine Kiendinger ! hoit . Hvad har Saul at giorc blandt Prophcter ? Cosmoligoreus . Gak strar bort , som jeg siger , 8 » ul inter r. on ! , . tl > s ? Petronius sagte . De sige : hvad har Saul at giore blandt Propheter ? Jeg kunde jo svare med en <b>vis</b> <b>Dame</b> : han ledte efter sin Herres Gslcr . Ha , ha , ha ! <b>Maalet</b> er ogsaa , gid jeg faae Skam ! Pernilles . Ha , ha , ha . ' Gaacr bort .

<h4>11447</h4>gaaed ham i at reformere Comoedier som den politifle Kandestøber i at reformere Staten , hvis Erempcl jeg haaber han ogsaa udi Catastrophen cftcrfslger , ved at opbrande eller auetioncrc de Skrifter , hvis Låsning har forledet ham til saadant , ligesom den Politiske Kandestsbcr giordc med sine politifle Naebtischcr eller Stokfister , der bragte ham i de Griller , at Hamborg maatte nsdvcndig gaac i Grunde , hvis han ikke blev Borgemester . I tiender , Hlonßieu > ' ! min Ekrive-Maade , og at jeg har et lidet Pund at komme Folk til at lec , naar jeg vil staac mig les ; her gives mig den rigeste Materie , jeg min Livs-Tid bar havt til laadant . Tank engang , hvor det vilde gaae Eder , om jeg vilde gisre Kepressnlier , om jeg vilde , som jeg efter saadan haanlig Provocation har Ret til , craminere hans Comoedier rgien ; thi om jeg vilde gisre det cndogsaa med den , som han kalder sit allerbedste Stykke , Den Seendis Blinde , skulde han selv forundre sig over , hvordan han var kommen paa de Tanker at agere Comcdie-Skrivcr . Jeg vilde suste , at jeg aldrig havde brugt nogen Kepl'essn!iei mod nogen af dem , der har angrebet mig tilforn ; jeg vil herefter strabc at overvinde den Passion , og derfor ikke hevne mig ved saadant Foretagende , men allene denne Gang , for at fere ham af vildfarelse , <b>vise</b> af nogle Poster , hvor ulykkelig ban har vcerct i at omstobe dette Stykke . Den lste Stsbning er udi Begyndelsen , hvor her gisrcs et Hvsteion protcron , og det , som fer var Fortale , bliver giort til Bagtale , efter hvilken Regel i or , ti < iueu , kan ikke vides . Vist nok er det , at Hlonziom- har ikke raadfsrt sig med som gav Regler i gamle Dage , ci heller som er Veiviser udi vore Tider . Med politi « , til hvis <b>Kundskab</b> jeg efter Reglerne har baned Vei i den hele fsrste Aet , begynder . Vlonzieni- strar sin omstsbte Comoedie , ' og derved synes at imitere det , som de Hollandske Linie-Dandsere kalder Venoenmssen i deres Nachspil , hvor Tilskuerne blive forvirret ved at see eller hsre noget , som de ingenlunde kan begribe , bvad det stal vcere , fsrcnd de har seet Enden ; eller visse tydste Romaner , der begvnde undertiden saaledes : , „ Gewis- , crwicderte der ungllickfclige Ritter Philander , " hvis Betydclse man ofte ikke faacr at vide , forend man kommer noget hen udi den tolvte Bog af Romanen ; da dog en Hoved- Regel udi Comoedier er , ikke at forestille uoget , uden man tilforn har givet Tilskuerne en tilstrakkelig <b>Kundskab</b> derom ; hvilket jeg nsic har iagttagcd i denne og alle mine andre Comoedier . Dette store IlMeion pioteion hedder hos ham , Ilunzieui- , en Reformation , som dog er tilstrækkelig uden Hielv at fordcerve et Stytte allene . Saaledes : Det fsrste Dakket trattes op. sees en Hoben Folk siddende ved et Bord , og En begynder Comoedien med disse Ord : „ Det vil give sig paa nceste Rigs-Dag , " uden at man vced , hvad det er for Folk , og hvad de vil sige . Den 2 den Stobning bestaaer derudi , at von Bremens <b>Kone</b> , Gedste . bliver giort til hans Svigcr-Moder , det er , af en myndig <b>Kone</b> , som har Part i hans ringe Stand saavelsom hans Hsihed , og derudover med fast ftsrre Virkning spiller sin Rolle udi den Comoedie , end Manden selv , bliver en Svigerfle , som hverken blir fattig eller rug , hverken hsi eller nedrig ved hans Forandring , og som han derforuden kunde efter de fleste SvigcriHSNneis Eremvel <b>vise</b> Dsrren , nnar ham lystede . derved er den anden Hoved-Person i Comoedien totnlitei- ruincred , og det , som var mest naturligt og anseeligt , giort tvungen og unaturligt ; imidlertid maa dette dog ogsaa hedde en Reformation . Den 3 die Stsbning bestaaer derudi , at af en enkelt Kiarlighed mellem Antoni Hiulmager og Engelke blir giort en dobbelt , som ingen Rapport har sammen , hvilket ! ogsaa er mod Neglerne , saasom en Comoedie derved gisres uforstaaelig , og betager Tilskuerne den en ret Comoedie bsr holde dem til ; thi Doubletter gior ligesaa slet Virkning i Comoedier , som god i Forkeering og < K For at fordcerve ogsaa denne Post , gior Hlunzieur Forlovelse imellem Kandestsberen og en Vsrstenbinder-Euke ; thi just de Ting , som udi sine Comoedier , holder for storstc Zirat , og beklager ikke at finde udi andre , nemlig dobbelte og tredobbelte Forlovelser , mangfoldige Varieteter , som ingen Overensstemmelse har med Historien , Mcengde af unyttige Personer , er just det , som fordcrrver alting . Saaledes mcerker jeg saavel udi denne , som i hans andre Historier , at han uden Betænkning gisr Folk dsve , blinde , krogryggede , halte , det er , sticerer Ore , Rase og Fodder af dem , allene for at faae Anledning til et < lo Ibe-iii- e , eller til noget , som han kalder Lon mut , skiondt de samme Personer ere ikke andet end lUuteni- « , som forsvinder , saasnart man faaer dem at see , og intet gisr til Comocdiens Udfald . Saaledes bliver udi denne nye Comoedie en Mand forestillet dsv , allene for at give Anledning til et fattigt Indfald , at han stal hsre Dotter for Doctor . Paa den Maade , for at spare Paviir og Vlcrk , kunde man kuns lade nogle ll « rle < sl , inß komme ind og vrange Munden , forst paa een Maade , og sige : Dette betyder den fsrste Aet ; siden paa en anden Maade , og sige : Dette betyder den anden Aet . Udi de allerbedste Comoedier , Mundur ! stal I finde stsrste Simplicitet i Historien . Et halv Qvintin don sen » gior bedre Virkning , end alle stige Optoger .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016101129001'target=_>Arndt, Johann, 1838, Sex Bøger om den sande Christendom</a><h4>1396</h4>du først hvad Christus er . Bil du kjende en god Frugt og Urt , da smag den , saa veed du det ; ligesaa maa du og smage Frugten af Livets Trce Christus , om du ret stal kjende ham . Smager du i Troen hans Ydmyghed , Sagtmodighed og Taalmodighed , saa ceder du af hans Frugter , saa finder du Hvile for din Sjæl , og bliver deelagtig i hans guddommelige Trost og Naade . Men gjor du det ikke , saa finder du ingen Hvile for din Sjæl . Thi Guds Naade og Trost kan ikke stinne ind i et trolost Hjerte , hvori Christi Sagtmodighed og Ydmyghed ikke er ; thi han giver ikkun de Ydmyge sin Naade , I Petr . 3,5 . Hvad gavner Christus detMenneste , som intet Samfund vil have med ham ? Nu have jo de , som leve i Syndens Morke , intet Samfund med Lyset , som er Christus , derfor er han dem intet nyttig ; thi saa siger Apostelen Johannes i bemeldte Sprog : Dersom vi sige , at vi have Samfund med ham , og vandre i Msrket , da lyve vi , og gjore ikke Sandhed . Det forklarer han ydermere i efterfølgende Capitel , sigende : <b>Mor</b> ker er forganget , og det sande Lys skinner nu ; hvo som siger : han er i Lyset , og hader sin Broder , han er endnu i Msrket ; hvo som elsker sin Broder , han bliver i Lyset , og der er ingen Forargelse i ham . Men hvo som hader sin Broder , er i Msrket , og omgaaes i Msrket , og veed ikke hvor han gaaer ; thi Msrket har forblindet hans Oine . Saa længe da et Menneske bliver i saadanne Synder , saasom i et førskrækkeligt Morke , da kan det ikke blive oplyst af Christo , det sande Lys , og komme til Guds rette <b>Kundskab</b> . Nåar man vil ret kjende Gud og Christum , da maa man vide , at Gud og Christus er idel Naade og Kjærlighed . Men Ingen kan vide , hvad Kjærlighed er , uden den , som selv har Kjærlighed , og over sig derudi . Saa kommer og den fuldkomne og rette <b>Kundskab</b> om de Ting , som stulle gjores , af Forfarenhed, og at man sligt har førsogt ; derfor kommer og Kjærligheds rette og fuldkomne <b>Kundskab</b> af Forfarenhed . Hvo som derfor ikke over sig i Kjærlighed , veed ikke hvad Kjærlighed er , om han endstjsndt taler meget derom . Christus er idel Kjærlighed , Ydmyghed , Sagtmodighed, Taalmodighed , idel Dyd og Fromhed . Hvo som ikke sver sig i dem , veed ikke hvad Christus er , og kjender ham ikke ret , i hvor meget han end taler om ham , og ncevner sig efter ham . § . 3. Guds Ord er idel <b>Aand</b> , hvo som derfor ikke lever og vandrer i Aanden , veed ikke hvad Guds Ord er , om han end noksaa meget taler

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012102924020'target=_>Holberg, Ludvig, 1856, Ludvig Holbergs samtlige Comedier i eet Bind</a><h4>7350</h4>Det er et underligt Slags Folk disse Poeter . Om en cerlig Mand giver et Vrefigen udi Tanker , trcekker man ham til Daarekisten ; men hos dem er Saadant Tegn til <b>Lcrrdom</b> og Dygtighed . Ellers fik jeg det Vers for godt Kisb . De Folk maa have nogen Frihed, efterdi de ere saa lidet interesserede ; thi hvad er dog tolv Skilling for saadan lcrrd Mand ? Jeg forssaaer ikke Verset , men kan dog see , at det er altid sine Penge vcerd . Nn stal jeg bilde min Husbond ind , at jeg har givet to Rigsdaler derfor , saa vinder jeg ved dette Wrinde elleve Mark og fire Skilliug . Jeg har kun liden <b>vis</b> Lsn , maa derfor ncere mig redelig ved Comnnssioner . Mangen brav Mand fsder <b>Kone</b> ogNsru dervedudiKisbenhavu ; thi hvorafstulde degode ellers leve ? Hvis jeg kunde flattere , som Jesper for min Herre , kunde jeg have de Smauscr og de Forceringer , som han faaer . Jeg stulde ikke rose mig selv ; men jeg er en cerlig Karl , der ikke kan sige Andet, end jeg mener . Naar min Hnsboud begyuder at tale om sin Manddom i den Vrabandste Krig , tier Peer ganske stille ; thi jeg veed , han har aldrig vcrret paa den anden Side af Beltet , end sige udi Braband , som ligger nogle tusinde Mile herfra . Nei . Wrlighed varer dog lcengst ; det hedder hos mig , som Hollcenderne siger : Thue das Recht nicht , und sty den Teuffel . Kunde jeg saa vel Tydst , som jeg kan Hollandsk , var Peer anden Mand , end han er ; thi jeg har tidt hsrt Alexander

<h4>20445</h4>Petronins ( sagte ) . De siger : hvad har Saul at giore blaudt Propheter ? leg kunde jo svare med en <b>vis</b> <b>Dame</b> : han ledte efter sin Herres 3 Esler . Ha , ha , ha ! <b>Maalet</b> er ogsaa , gid jeg faae Skam , Pernilles. Ha , ha , ha ! ( Gaaer bort ) .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012102924019'target=_>Holberg, Ludvig, 1853, Ludvig Holbergs samtlige Comedier i eet Bind</a><h4>20525</h4>Petronins ( sagte ) . De siger : hvad har Sanl at gisre blandt Propheter ? Jeg knnde jo svare med en <b>vis</b> <b>Dame</b> : han ledte efter sin Herres LEsler . Ha , ha , ha ! <b>Maalet</b> er ogsaa , gid jeg faae Skam , Pernilles. Ha , ha , ha ! ( Gaaer bort ) .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016100729002'target=_>Arndt, Johann, 1844, Sex Bøger om den sande Christendom</a><h4>921</h4>kan stee , uden saa er at man opstaaer af Syndessvnen , det er , af Übodfcrrdighed , Sikkerhed og Ugudelighed . Derfor staaer der og , Av . G . , 2 , 38. : Omvender eder , saa stulle I faae den Hclligaands Gaver . Joh . 14,17 . : Verden kan ikke annamme Sandheds <b>Aand</b> ; men hvad er Verden andet end at leve ugudelig ? Og hvad er det ogsaa Herren siger Match . 7 , 20. : Paa deres Frugter stulle I kjende dem ? det er ikke andet end , at sande og falske Christne kunne kjendes af deres Levnets Frugter , men ikke deraf , at de get raabe : Herre , Herre . Thi de falske Christne skjule og besmykke sig med den rene Lårdoms Skin , saasom med Faareklcrder , alligevel de i Hjertet ere intet mindre end sande Christne . Vel bor ingen domme om Ens <b>Lcrrdom</b> ester Levnet , og indbilde sig , at Lårdommen derfor er vrang og falst , fordi den findes hos dem , der fore et stemt Levnet , saasom Gjendobere og Papisterne vrangeligen deraf domme ilde om vor , 3 oerdom , fordi saa mange iblandt os , ligesaavelsom iblandt dem , leve ilde . § 5. Thi det folger ingenlunde , at Lårdommen derfor maa være usand og . falst , fordi at Folk , som den have , ined deres ugudelige Levnet forholde sig ilde ; thi da maatte ogsaa Christus og A ' postlerne have lært det som var falskt og usandt , efterdi der og i deres Tid vare mange stemme Mennesker . Ikke dcsmindre er dog det onde Levnet en Prove paa Personen selv , hos hvilken det sindes , og vidner , om han er en falst eller en sand Christen , om han lever som han larer , om han troer rettelig , eller han <b>mor</b> anderledes og imod Troen . Om saadanne siger den Herre Christus , at de ere falske Christne , ufrugtbare onde Tra-er , hvilke derfor henhore til Ilden , Match . 7,19 . Omsider, kort at sige , da er det den sande Troe , der virker kraftig ved Kjærlighed , Gal . 5 , 6. , hvorved Mennesket bliver til et nyt Kreatur , fodes paa ny , og bliver forenet med Gud , hvorved Christus boer i os , Eph . 3 , 17. , lever og arbeider i os , hvorved Guds Rige opbygges i os , hvorved den Hellig-Aand renser og oplyser vort Hjerte . Derom vidne adskillige herlige Sprog i Skriften , som 1 Cor . 6 , 17. : Hvo som hanger ved Herren , eller holder sig til ham , han er een Nand med ham . Hvad er det at vare een

<h4>3406</h4>vor < Konge givet os sin <b>Aand</b> , som kommer vore Ekrobeligheder til Hjælp , og træder frem for os med usigelige Sukke , " Rom . 8,26 . Her bliver tillige Fuldkommenheden og Umuligheden modsagt ; thi at Paulus siger Eph . 4 , 13. : „ At vi Alle stulle mode Troens og Guds Sons Erkjendelses Enig . hed , til en fuldkommen Mand , til Christi fuldkomne Værtes Maade , er ikke at førstaae om vort Levnet , men om den fuldkomne <b>Lærdom</b> , som forer os til Salighed , hvoraf intet er forholdet os , men alt Guds Raad aabenbaret , hvorledes vi stulle blive salige , paa det vi ikke stulle lade os forfore , som Apostelen selv forklarer sig , at „ vi ikke mere stulle være Born , og lade os omfores af hvert Lærdoms Veir . " Saa siger han og Eol . 1 , 28. : „ Vi paaminde hvert Menneske , og lære hvert Menneske i al Viisdom , at vi kunde fremstille hvert Menneske fuldkommen i Christo Jesu . " Det er vi lære hvorledes i Ehristv Jesu alene er den fuldkomne Retfærdighed og fuldkomne Vei til Salighed . Desligeste 2 Tim . S , 16.17 . : „ Den ganske Skrift er inblcest af Gud , og er nyttig tll <b>Lærdom</b> , til Overbeviisning , til Rettelse , til Optugtelse i Rctfcerdighed , at det Guds Menneske stal være fcerdlgt tilbereder til al god Gjerning ; " det er , Guds er en fuldkommen <b>Lærdom</b> og Vei til Salighed , oq <b>mor</b> Mennesket stikket til alle gode Gjerninger , som behage Gud , saa man ikke har fornoden at optanke eller foie nogle nye Lårdomme eller Gjerninger dertil . § 4. Det tredie ' Slags , som af idel Ondstab og Modvilllgyed laste og foragte den <b>Lærdom</b> om den sande Poe- og christelige Levnet , de laste Cbristi hellige Lidelse , og spotte Naadens <b>Aand</b> og Christi Erempels Efterfolgelse , de maae vel betænke sig , hvad de ville svare den Herre Christo , da han siger , Luc . 9 , 23. : „ Hvo som vil komme efter mig , stal fornegte sig selv , og tage sit Kors op daglig , og solge mig . " Item , „ Dersom nogen kommer til mig , oa hader ikke nt eget Liv , han kan ikke være min Discipel " Luc 14,26 . ; item Matth . 11,29 . : „ Lcerer af mig , thi jeq er sagtmodig og ydmyg af Hjertet . " Item , hvad de ville svare Apostelen Peder , naar han siger : „ Ehnstus efcerloo os et Erempel , at stulle efterfolge hans Fodspor , " 1 P.rr 2,21 . , og Evangelisten Johannes , naar han siger 1 Eph

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012101624028'target=_>Holberg, Ludvig, 1876, danske Skueplads, eller Holbergs Comedier</a><h4>2648</h4>Raifon af mig til det , som all Verden ittc kand give Raison til . Hvorfor heder een , som er fod stievbenet . Velbaaren ? Hvorfor heder een Hoylcerd i Dag , som neppe kunde stave i Gaar ? Hvorfor strider man Fransk Udskrift paa et Brev , fom gnaer imellem Slagelse og Ringsted ? Hvorfor kand man ikke hitte paa et Ord i Steden for Franco ? Og tusinde andre Ting , som mig er umueligt at opregne . Peer . Det er sandt , som du siger . Jeg er kun en fattig Tienere , og kand dog begribe , at det er Galstab . Jeg tcenter paa , om en Vexel ikke kunde vcere lige god , hvis det Ord Valuta blev strevet paa Dansk . Jens . Jeg mener jo . Ligesom din Herre er ikke et Haar bedre , siden han lod sig kalde Von Tyboe , end da han heedte flet og ret Tyboe . Men man maa ikke altiid gruble saa meget , den ene man giore som den anden og lade sig noye dermed , at det er Landets Mode . Peer . Ja det maa vcere Landets Mode eller ey , saa lod jeg mig dog aldrig kalde Von Peer , om jeg blev til noget . Jens . Ney , du Nar ! du maatte tage dig et Tilnavn forst . Peer . Hvor Fanden skulde jeg fane det fra ? Min Fader heedte kun slet og ret Peer , min Far-Far og Olde-Far ligesaa , jeg kand fast opregne mine sexten Ahner af lutter Peerer , som aldrig haver havt videre Tilnavn . Jens . Ach derfor kunde du vel hitte paa noget . Hvor est du fod ? Peer . I Europia . Jens . Ja det er troeligt nok ; jeg kand hsre paa dit <b>Maal</b> , at du est en Europcrer og ingen Polakker . Men paa hvad Sted udi Landet est du fod ? Peer . Udi Kiobenhavn . Jens . I hvad Gade udi Kiobenhavn ? Peer . I Klceboerne . Jens . Ja saa stal du lade dig kalde Peter eller Peiter von Klceboe . Peer . Jeg faner vist Skam , jeg staner her saa ; jeg har saa meget at forrette for min Herre . Han vil gisre Vers til en <b>Pige</b> paa Fransk , som han er forliebt udi ; men som han ikke er studeret selv , har han bedet mig at oplede en Poet , som i Hast kan smede noget sammen for ham . Jens . Jeg stal <b>vise</b> dig een , som cnsr Vers for en billig Priis . Peer . Hvor boer han ? Jens . Han boer udi Aabenraae , nest ved Guldsmeden . Peer . Hvad er hans Navn ? Jens . Jeg troer ikke , han har noget Navn ; jeg har aldrig hort , at han kaldes anderledes end Poeten i Aabenraae . P e er . Jeg veed jo , Poeter bliver doble som andre Mennesker . Jens . Det stal jeg ikke egentlig kunde sige dig ; nok er det , at du sporger efter Poeten i Aabenraae .

<h4>2683</h4>Det er underligt Slags Folk disse Poeter . Om en anden cerlig Mand giver et Orefigen udi Tanker , trcetker man ham til Daare-Kisten ; men hos dem er saadant Tegn til <b>Lcerdom</b> og Dygtighed . Ellers fik jeg det Vers for got Kiob . De Folk maa have nogen Frihed , efterdi de ere saa lidet interesserede; thi hvad er dog 12 h . for saadlln lcerd Mand ? Jeg forstaaer ikke Verset , men land dog see , at det er altiid sine Penge vcerd . Nu stal jeg bilde min Hosbond ind , at jeg har givet 2 Rixdlr . derfor , faa vinder jeg ved dette Mrende 11 Mart og 4 h . Jeg haver kun liden <b>vis</b> Lsn , maa derfor ncere mig redelig ved Commissioner . Mangen brav Mand foder <b>Kone</b> og Born derved udi Kiobenhavn : thi hvoraf stalde de gode Folk ellers leve ? Hvis jeg kunde flattere , som Jesper gior for min Herre , kunde jeg have de Smautzer og de Forceringer , som han saaer . Jeg skulde ikke roese mig selv ; men jeg er en cerlig Karl , der ikke kand sige andet , end jeg mener . Naar min Hosbond begynder at tale om sinMcmddom

<h4>3728</h4>Sganarell . Ikke om du vilde give mig all Verdens Guld . Jeg gir ingen Rapport til » ndre ) end til Polidorus felv . Dorothea . Er Leanders Hcer meget stcerk ? Sganarell . Det veed jeg ikke ; men- - - Dorothell . Hvad veed du da ? Deri stal jo den hele <b>Kundskab</b> bestaae . Er de da snart fcerdig med deres Udrustning ? Sganarell . Det stal jeg ikke heller kunde sige ; men jeg veed i det Ovrige alting baade til Lands og Vands . Dorothea . Mig synes , at alt det Ovrige land belobc sig til siet intet . Sganarell . Ney hor , hun vil min Troe ogsaa raisonnere i Kriigs-Sager . Dorothea . Han gior sig saa mcend allerede en Mine , som han var en Minister . Jeg veed , du land jo sagte sige mig noget . Sganarell . Ikke om Cabeljouen og Flynderne engang . Dorothea . Hvad er det for Snak ? Sgllnarell . Ikke om den 40 Aars lomfruedom engang . Dorothea . Ney hor engang , hvor forblommet han taler . Det er scert med disse Dosmere , at naar dem noget blir betroed , saa bilder de sig ind , at all Verdens Viisdom ligger stiult derunder , og gior store Hemmeligheder af Bllgllteller , ligesom en <b>vis</b> Rcmds- Herre , der cergrede sig snart ihiel , eftersom han udi Sovne havde sagt sin <b>Kone</b> , at Borgemesteren havde spildt Blcek paa sin Klud paa Raadhuset . Sllaledes gaaer det og med denne ; thi jeg troer , han lod sig for.hcenge , end sagde , i hvad Klcede-Dragt han havde seet een af Fiendens Folk . Det var at snste , at een vilde tage sig for at skrive en Comoeoie over Foll cu deslige Caracterer , over saad°n u-rnneUg Politiqve og affectered Tausyed. Men der kommer een med et hvidt Baand paa Hatten . Lad os lobe .

<h4>4186</h4>Pernille . Men siig mig dog hvad det er , min Balsombysse ! Heniich . Han er min Papa , det er sandt , som jeg er all Resp eet skyldig ; men jeg gir ham Dievelen . Pernille . Hvad har han skreven , min Engel ? Henrich . Han maa vide parcli , at jeg Diadl « ni ' empoi-ts har traad mine Bornestoe . Pernille . Ach dslg det dog ikte lcengere for mig . Henrich . Jeg vil doe paa at det er min Sosters , Froyken Fikkes Intriguer . Ja jeg vil vcere et Carnnlli-Beest , om det ikke er . Pernille , tzvnd er det for Intriguer ? Henrich . Men est du ina Bc > sur , sna stal jeg <b>vise</b> dig , nt jeg er din mon trsrs , petits Oauaills ! Pernille . Siig mig det dog , min sode Sioel ! eller jeg doer . Henrich . Veedst du hvad det er , min Poppe ? Min Papa skriver mig til , at han har hort med Hierte-Sorg , at jeg lober om og gior til <b>Fruentimmer</b> udi Byen . ? ar6i mon jeg ikke er af den Alder , at jeg veed hvad jeg gior ? I saadnne Sager spor jeg ingen uden mit eget Hierte til Raads . Pernille . Det mener jeg ogsaa . Henrich . Jeg er alt en Karl , der lader mig rage 3 Gange om Ugen . Pernille . Men hvnd skriver han videre , mit Hierte ! efterdi du blir faa vreed ? Henrich . Jeg er en Karl , som forstaaer mit Fransost og Italiensk som mit Moers- <b>Maal</b> , og skulde ikke vide hvad min Velfcerd er ? Men I stal faae en lang Ncese , rncm onkr ? apa ! Pernille . Hvad er din Papas Forscet dn med dig , min Skat ? Henrich . Han er god nok , min Dukke ! for sin egen Person ; men han lar sig forfore . Det er ikke at bestride hvad Fynter der er hos min Soster , Froyken Fikke . Pernille . Hvnd gior hun da ? Henrich . Jeg veed nok hvad din Ncese kloer efter , nnadige Froyken ! Men du veedst ikke , hvilke Venner jeg har til Hove , og at Kongen offentlig har sngt om mig over Taffel : Den unge Herre er noget Extraordinaire hos . Hor , mm Dukke ! min Pnpa har stuttet Partie mellem nng og en Froyken paa Landet , oq Ml at jeg med det forste stal have Bryllup med hende . Pernille . Ach Magdelone ! holdt mig , M besvimmer . " '

<h4>7690</h4>Petronius . Jo meer jeg beskuer ham , lo meer han ligner hende . Pernille . Lad ham gane reent bort : thi ieg har andet af Vigtighed at tale om . Cosmoligoreus . Gak kun hiem til dine Forretninger , Petroni ! til jeg kalder paa dig . Petronius . Ach Herrel jeg har og Lyst at tale med denne Fremmede ; han ligner een af mine Kiendinger . Pernille . Gane bort , naar jer Herre befaler jer . Sagte . Gid du faae en U-lykke med dine Kiendinger . cHsyt.i Hvad har Saul at giore blant Propheter ? Cosmoligoreus . Gak strax bort , fom leg siger . Hvi6 Bs , ul iutsr I » i-op1i6tg , 3 ? Petronius sagte . De sige : Hvad har Saul at giore blant Propheter ? Jeg kunde lo fvare med en <b>vis</b> <b>Dame</b> : Han ledte efter sin Herres Wsler . Ha ha ha ! <b>Maalct</b> er ogsaa, gid jeg faae Skam , Pernilles . Ha ha ha ! Gaaer boit .

<h4>7704</h4>der udi philosophiste Skoler og Collegier oplceres udi Vildfarelser , og faaer en falsk Idee om Philosophie , faa de bilde sig ind , at den bestaaer udi Kaabe , Skiceg , Egensindighed eller udi puur Theorie , stiont de rette Lceremestere bor ikke saa meget beflitte sig paa at giore deres Disciple lcerde som at giore dem gode og dydige . Nu er han kommen under anden Information , som er end meer gall end den , han har faaet i sin Skolegang . Leander . Monsr . Jeronimus mener den Pragiste Philosophie ? Ha ha ha ! Saa nt jeg tcrnler derpaa , maa ieg forundre mig over denne <b>Piges</b> Capacitet . Jeg var alleene bange for at hun outrerede Tingen for meget . Jeronimus . leg ikke ; thi naar man forst har indprentet nogen hoye Tanker om sin Person og <b>Lcrrdom</b> , tand man siden let faae ham at troe hvad man vil , helst naar det flatterer eens egen Tilboyelighed . Thi ingen <b>Lcrrdom</b> kunde komme Cosmollgoreo meer til Pas end den , fom indeholdes udi det Pragiste Morale , hvorudi Rang-Syge giorcs til en honete Ambition og til en ptnlosophist Dyd . Leander . Men hvad stal man nge om den 14 de Artikel , som tillader en Mand at ligge hos sin <b>Pige</b> 2 Ncetter om Ugen uden at bedrive Synd ? Jeronimus . Hun forte dette gandske vel ud , forst ved at foregive , at saadant stede efter den store Pragiste Doctors Prcescription , derncest ved at <b>vise</b> , at saadant tand stee uden Synd , naar Intentionen er god , og Sicelen er reen . Thi hvor u-rimelig saadan <b>Lcerdom</b> end er , saa seer man dog , at den antages af alle ? kn » , tioiB . Leander . Pigen er stemplet , og sortiener at doctoreres . leg ansaae med Forundring, hvorledes hun redede sig ud af den Fristelse , som Tieneren bragte hende udi . Jeronimus . Ja det samme var den artigste Scene . leg haaber , at hun vil lykke' ligen fuldende Tingen , fom hun har saa vel begyndt . Leander . Hvad mon hun videre vu ' giore ? Jeronimus . Det vil hun ikke Uge ; ieg tor ey heller giore mig saa dristig at sporge den store Pragiste Doctor derom , vil derfore lade det Ovrige lomme an paa hende alleene . leg stal ellers give Monsr . Leander videre Underretning derom og advare ham , naar Tiid er at lade sig indfinde . Leander . Adieu , Hr. Jeronimus ! leg recommenderer mit Anliggende paa det Beste . Jeronimus . Han maa ilke tvivle om t min Velvillighed . Adieu !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012101624017'target=_>Holberg, Ludvig, 1876, Holbergs Comedier</a><h4>2552</h4>Rcnson af mig til det , fom all Verden ikte kand give Rcnson til . Hvorfor heder een , som er fod stievbenet , Vclbaaren ? Hvorfor heder een Hoylcerd i Dag , fom neppe kunde stave i Gaar ? Hvorfor strivcr man Fransk Udskrift paa et Brev , som gaaer imellem Slagelse og Ringsted ? Hvorfor kand man ikke hitte paa et Ord i Steden for Franco ? Og tusinde andre Ting , som mig er umueligt at opregne . Peer . Det er sandt , som du siger . Jeg er knn en fattig Tienere , og kand dog begribe , at det er Galstab . Jeg tcenkcr paa , om en Vexel ikke kunde vcere lige god , hvis det Ord Valuta blev strevet paa Danst . Jens . Jeg mcnerjo . Ligesom din Herre er ikke et Haar bedre , siden han lod sig kalde Von Tyboe , end da han heedte siet og ret Tyboe . Men man maa ikke altiid gruble saa meget , den ene maa giore som den anden og lade sig noye dermed , at det er Landets Mode . Peer . Ja det maa vcere Landets Mode eller ey , saa lod jeg mig dog aldrig kalde Von Peer , om jeg blev til noget . Jens . Ney , du Nar ! du maatte tage dig et Tilnavn forst . Peer . Hvor Fanden stulde jeg fane det fra ? Min Fader heedte kun slet og ret Peer , min Far-Far og Olde-Far ligesaa , jeg kand fast opregne mine sexten Ahncr af lutter Peerer , fom aldrig haver havt videre Tilnavn . Jens . Ach derfor kunde du vel hitte paa noget . Hvor est du fod ? Peer . I Europia . Jens . Ja det er troeligt nok ; jeg kand hore paa dit <b>Maal</b> , at du est en Europceer og ingen Polakker . Men paa hvad Sted udi Landet est du fod ? Peer . Udi Kiobenhavn . Jens . I hvad Gade udi Kisbenhavn ? Peer . I Klceboerne . Jens . Ja faa stal du lade dig kalde Peter eller Peiter von Klceboe . Peer . Jeg faaer vist Skam , jeg staller her faa ; jeg har faa meget at forrette for min Herre . Han vil giore Vers til en <b>Pige</b> paa Fransk , fom han er forliebt udi ; men som han ikke er studeret selv , har han bedet mig at oplede en Poet , som i Hast kan smede noget sammen for ham . Jens . Jeg stal <b>vise</b> dig een , som gior Vers for en billig Priis . Peer . Hvor boer han ? Jens . Han boer udi Aabenraae , nest ved Guldsmeden . Peer . Hvad er hans Navn ? Jens . Jeg troer ikke , han har noget Navn ; jeg har aldrig hort , at han kaldes anderledes end Poeten i Aabenraae . P e er . Jeg veed jo , Poeter bliver dobte som andre Menn ester . Jens . Det stal jeg ikke egentlig kunde sige dig ; nok er det , at du sporger efter Poeten i Aabenraae . Peer . Tak for god Underretning ! Jeg vil gaae lidt ind til Christoffers og drikke en Potte 01 l forst , siden vil jeg hen llt lede den gode Poet op. Adios !

<h4>2588</h4>Det er underligt Slags Folk disse Poeter . Om en anden cerlig Mand giver et Orefigen udi Tanker , trcekker man ham til Daare-Kisten ; men hos dem er faadant Tegn til <b>Lcerdom</b> og Dygtighed . Ellers sil jeg det Vers for got Kiob . De Folk maa have nogen Frihed , efterdi de ere faa lidet interesserede; thi hvad er dog 12 tz . for saadan lccrd Mand ? Jeg forstaaer ikke Verset , men land dog see , at det er altiid sine Penge vcerd . Nu stal jeg bilde min Hosbond ind , at jeg har givet 2 Rixdlr . derfor , saa vinder jeg ved dette LErende 11 Mark og 4 tz . Jeg haver kun liden <b>vis</b> Lon , maa derfor ncere mig redelig ved Commissioner . Mangen brav Mand foder <b>Kone</b> og Born derved udi Kiobenhavn ; thi hvoraf skulde de gode Folk ellers leve ? Hvis jeg kunde flattere , som Jesper gior for min Herre , kunde jeg have de Smautzer og de Forceringer , fom han faaer . Jeg skulde ikke roese mig selv ; men jeg er en cerlig Karl , der ikke tand sige andet , end jeg mener . Naar min Husbond begynder at tale om sin Manddom

<h4>3712</h4>Sganarell . Ikke om du vilde give mig all Verdens Gnld . Jeg gir ingen Rapport til end til Polidorus selv . Dorothea . Er Leanders Hcer meget stcerk ? Sganarell . Det veed jeg ikke ; men- - - Dorothea . Hvad veed du da ? Deri stal jo den hele <b>Kundskab</b> bestane . Er de da snart fcerdig med deres Udrystning ? Sganarell . Det stal jeg ikke heller kunde sige ; men jeg veed i det Ovrige alting baade til Lands og Vands . Dorothea . Mig synes , at alt det Ovrige land belobe sig til slet intet . Sganarell . Ney hor , hun vil min Troe ogsaa raisonnere i Kriigs-Sager . Dorothea . Han gior sig saa moend allerede en Mine , som han var en Minister . Jeg veed , du land jo sagte sige mig noget . Sganarell . Ikke om Cabeljouen og Flynderne engang . Dorothea . Hvad er det for Snak ? Sganarell . Ikke om den 40 Aars lomfruedom engang . Dorothea . Ney hor engang , hvor forblommet han taler . Det er soert med disse Dosmere , at naar dem noget blir betroed , saa bilder de sig ind . at all Verdens Viisdom ligger skiult derunder , og gior store Hemmeligheder af Bagateller , ligesom en <b>vis</b> Raads- Herre , der oergrede sig snart ihiel , eftersom han udi Sovne havde sagt sin <b>Kone</b> , at Borgemesteren havde spildt Blcek paa sin Klud paa Raadhuset . Saaledes gaaer det og med denne ; thi jeg troer , han lod sig for hoenge , end sagde , i hvad Kloede-Dragt han havde scet een af Fiendens Folk . Det var at snste , at een vilde tage sig for at skrive en Comoedie over Folk af deslige Caracterer , over saadan u-rimelig Politiqve og affcctered Taushed. Men der kommer een med et hvidt Bllllnd paa Hatten . Lad os lobe .

<h4>4159</h4>Pernille . Ach hvad er det , mit halve Hierte ? , ° tzenrich . Det var jo forbandet . Han l vil true mig dertil ; staaer det ikke saa i Slut- 3 ningen ? Jo vist : H.ntrsrQ6nt LxNlainckigimc » d6Btialsin6llt , antrsiQLiit . Men jeg er Karl c for min tzat . Pernille . Ach hvad er det do.g , mm Snut ? tzenrich . Om alt andet siaaer feyl , saa i har jeg dog mine to Herregaarde , som er min i Moderne-Arv . ... Pernille . Men sug nug dog hvad det t er , min Balsombysse ! . , c Henrick ) . Han er mm Papa , det er i sandt , som jeg er all Respect skyldig ; men ieg l gir ham Dievelen . . < Pernille . Hvad har han skreven , mm , Engel ? . - ' Henrick ) . Han maa vide parcki , at jeg Diable » 10. ' smporw har traad mine Bornestoe . Pernille . Ach dolg det dog ikke lamgere ! for mig . , tzenrich . Jeg vil doe paa at det er min Sosters , Froyken Fikkes Intriguer . Ja ieq vil vcere et Carnalli-Beest , om det ikke er . Pernille . Hvad er det for Intriguer ? Henrich . Men est du mg . noenr , saa stal jeg <b>vise</b> dig , at jeg er din mon trers , pLtits OankillL ! Pernille . Siig mig det dog , min sode Sicel ! eller jcg doer . tzenrich . Veedst du hvad det er , mm Poppe ? Min Papa skriver mig til , at han har hort med Hierte-Sorg , at jeg lober om oq gior til <b>Fruentimmer</b> udi Byen . mon jeg ikke er af den Alder , at jeg veed hvad jeg gior ? I sandane Sager spor jeg ingen uden mit eget Hierte til Raads . Pernille . Det mener jeg ogsaa . Henrich . Jeg er alt en Karl , der lader mig rage 3 Gange ' om Ugen . Pernille . Men hvad skriverhcm videre , mit Hierte ! efterdi du blir saa vreed ? Henrick ) . Jeg er en Karl , som sorstaaer mit Fransost og Italiensk som mit Moers- <b>Maal</b> , og skulde ikke vide hvad min Velfcerd ev ? Men I stal faae en lang Ncefe , mon Her ? 2 , p » , ! Pernille . Hvad er din Papas Forscet da med dig , min Skat ? Henrich . Han er god nok , min Dukke ! for sin egen Person ; men han lar sig forfore . Det er ikke at beskrive hvad Fynter der er hos min Soster , Froyken Fikke . Pernille . Hvad gior hun da ? Henrich . Jeg veed nok hvad din Ncese kloer efter , naadige Froyken ! Men du veedst ikke , hvilke Venner jeg har til Hove , og at Kongen offentlig har sagt om mig over Taffel : Den unge Herre er noget Extraordinaire hos . Hor , min Dukke ! min Papa har stuttet Partie mellem mig og en Froyken paa Landet , og vil , at jeg med det sorste stal have Bryllup med hende . Pernille . Ach Magdelone ! holdt mig ,

<h4>7933</h4>Petronius . Jo meer jeg beskuer ham , jo meer han ligner hende . Pernille . Llld ham gaae reent bort ; thi jeg har andet af Vigtighed at tale om . Cosmoligoreus . Gak kun hiem til dine Forretninger , Petroni ! til jeg kalder paa dig . Petronius . Ach Herre ! jeg har og Lyst at tale med denne Fremmede ; han ligner een af mine Kiendinger . Pernille . Gaae bort , naar jer Herre befaler jer . Sagte . Gid du faae en U-lykte med dine Kiendinger . lHsyt . ) Hvad har Scml at giore blant Propheter ? Cosmoligoreus . Gak strax bort , som jeg siger . Hvi6 Baul intsr ? i-oxli6taB ? Petronius sagte . De sige : Hvad har Sau ! at gisre blant Propheter ? Jeg kunde jo svare med en <b>vis</b> <b>Dame</b> : Han ledte efter sin Herres Msler . Ha ha ha ! <b>Maalet</b> er ogslla, gid jeg faae Skam , Pernilles . Ha ha ha ! Ganer bort .

<h4>7947</h4>der udi philosophiste Skoler og Collegier oplæres udi Vildfarelser , og faaer en falsk Idee om Philosophie , saa de bilde sig ind , at den bestaaer udi Kaabc , Skiceg , Egensindighed eller udi puur Theorie , stiont de rette Lceremestere bor ikke saa meget beflitte sig paa at giore deres Disciple lcerde som at giore dem gode og dydige . Nu er han kommen under anden Information , fom er end meer gall end den , han har faaet i sin Skolegang . Leander . Monsr . Jeronimus mener den Pragiste Philosophie ? Ha ha ha ! Saa lit jeg tcenker derpaa , maa ieg forundre mig over denne <b>Piges</b> Capacitet . Jeg vllr allcene bange for at hnn outrerede Tingen for meget . Jeronimus . leg ikke ; thi naar man forst har indprentet nogen hoye Tanker om sin Person og <b>Lcerdom</b> , kand man siden let faae ham at troe hvad man vil , helst naar det flatterer eens egen Tilboyelighed . Thi ingen <b>Lcerdom</b> kunde komme Cosmoligoreo meer til Pas end den , som indeholdes udi det Pragiste Morale , hvorudi Rang-Syge giores til en honete Ambition og til en philosophist Dyd . Leander . Men hvad stal man sige om den 14 de Artikel , som tillader en Mand at ligge hos sin <b>Pige</b> 2 Ncelter om Ugen uden at bedrive Synd ? Jeronimus . Hun forte dette gandske vel ud , forst ved at foregive , at saadant stede efter den store Pragiste Doctors Proescription , derncest ved at <b>vise</b> , at saadant kand stee uden Synd , naar Intentionen er god , og Sicelen er reen . Thi hvor u-rimelig faadan <b>Lcerdom</b> end er , saa seer man dog , at den antages af alle ? 2 , i > , ktieiB . Leander . Pigen er stemplet , og fortiener at doctoreres . leg ansane med Forundring, hvorledes hnn redede sig ud af den ' Fristelse , som Tieneren bragte hende udi . - Jeronimus . Ja det samme var den ' artigste Scene . Jeg haaber , at hun vil ' ligen fnldendc Tingen , fom hun har faa vel begyndt . Leander . Hvad mon hun videre VU ' giore ? Jeronimus . Det vil hun ikke sige ; jeg tor ey heller giore mig saa dristig at sporge den store Pragiste Doctor derom , vil derfore lllde det Bvrige komme lln plla hende alleen « . leg stal ellers give Monsr . Leander videre Underretning derom og advare ham , naar Tiid er at lade sig indfinde . Leander . Adieu , Hr. Jeronimus ! leg ' recommendercr mit Anliggende paa det Beste . leronimns . Han maa ilke tvivle om l min Velvillighed . Adieu !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013042608062'target=_>Luther, Martin, 1858, Kraft- og saftfulde Kjerne af de evangeliske Sandheder, udtagen af Dr. Martin Luthers Kirke- og Huus-Postiller</a><h4>7458</h4>De derimod paa den anden Side , hvilke Prcedike- Embedet ei er anbefalet , men som behove Underviisning og <b>Lcerdom</b> , de , siger jeg , stal vcere Sonner , og lade sig <b>vise</b> , fore , erncere og pleie , og stille sig an mod deres Lcerere , som et fromt Barn mod sin <b>Moder</b> . Kjcerligheden er vel ei saa stor i Born til Moderen , som i Moderen til Born , efter det almindelige Ordsprog: Kjcerligheden gaaer mere nedad , end opad ; saa driver dog Naturen fromme Born til at cere og tjene deres Forceldre i alle Ting . Naar det saaledes gaaer til imellem <b>Moder</b> og Son , Prcedikantere og Lcerere , saa gaaer det vel til .

<h4>10190</h4>pcde imod Braadden , saa revocerede dog Agricola siden derefter ; imidlertid han sacwel som Anabaptisterne med deres rasende <b>Lcerdom</b> svcermede , som greb alt mere og mere om sig , har Luther , for saavidt muligt , det kunde staae i hans Magt , at forebygge alt for stor Forfsrelse, udgivet en skriftlig Advarsel saavel for Antinomis og Agricola , som for Anabaptisterne . Da denne Urolighed meest rasede , saa laae dog den vankundige Hobs Nsd Luther paa Hjertet , at de sadde i saa dyb og stor Vanvittighed , og for at give dem den fornodne Oplysning til deres Sjceles evige Velfcerd og Salighed, har han forfcerdiget de tvende gyldne Boger , sin liden og store Catechismus , som han Aar 1529 " udgav i Trykken , hvilket Luthers Arbeide var desto nodvendigere, som Folk i Pavedsmmet stak i den dybeste Vankundighed, og hartad aldrig Horte de ti Bud , Troens Artikle , Fader-Vor , eller Daaben at forklares , og var derfor en beklagelig Tilstand paa de Tider med de Catechetiffe Kvelser , og som Mathesius bevidner , saa mindes han ikke at have seet nogen skreven eller trykt Forklaring over Bornelcerdommen i Pavedsmmet , da han dog fra Ungdommen op havde lcest alle Legender og St . Brigittce Nonner , ja den redelige Mathesius forsikkrer tillige helligen , at han aldrig i sin Livstid , saavidt han kunde erindre sig , har hort nogen i Kirkerne eller Skolerne at tale om Absolutionen og den Trost som man ved Christi Legems og Blods Nydelse kunde bekomme , forend han kom til Wittenberg , men i Skolerne lcestcs i Fasten om Skriftemaal og det ene Slags i Nadveren ; men det som ellers gav Luther den fornemste Anledning til hans store og lille Catechismi Udgave var den store Vanvittighed , som Luther formcerkede baade blandt Lcererne og Tilhorerne paa sin Visitation , han paatog sig , thi faa vidste noget om Catechismo og deres Bornelcerdom. Da Luther i sin Visitation overhorte de arme Bonder , og med stor Taalmodighed baade tilspurgte dem om deres Christendom , faa og underviste dem deri , saa forefaldt en scerdeles artig Historie . Dr. Luther bad en arm sachsist Bonde at oplcese Troens Artikler. Han begyndte derpaa i sit Sprog at lcese : " Jeg troer paa Gud den Almcegtige " . Dr. Luther spurgte ham videre : Hvad det var at vcere almcegtig ? Den gode eenfoldige Mand svarede : " Jeg veed det ikke " . Ja min Mand , sagde Dr. Luther , jeg og alle Lcerde vide i el heller , hvad Guds Kraft og Almagt er ; men troe « du kun i Eenfoldighed , at Gud er din kjcere og trofa- l ste Fader , som vil , kan og veed , som den allerklogeste l Herre , at hjcelpe din <b>Kone</b> og Born . Korte , eenfol- > dige , fyndige , ordentlige og gode , ja uforlignelig fijon- ! l ne ere disse tvende Boger , og den saliggjsrende <b>Lcerdom</b> i deri reent og med saa hoi en <b>Aand</b> foredraget , af den 5 ltlle Catechismus efter Hr. Seckendorfs , den store og k gudfrygtige Statsmands Dom neppe har sin Lige , og f er sandelig en gylden Bog . Hvad for en utrolig Frugt f og Nytte denne Borne-Bog har udrettet , saavel i Kir- ( 5 ker som Skoler , er kundbart , og deri kan man oiensyn- f lig spore , at Gud rigelig og overflsdig har velsignet d dette Luthers Arbeide . Mathesius bevidner , at Dr. Pom- 2 mer har saa hoit agtet den lille Catechismus , at han

<h4>10308</h4>s . om er esterfslgende : Der blev een , for besat , udgiven n <b>Pige</b> , 18 Aar gammel , fsrt til Luther ; han bad hende d bede Troen , hvilket hun ogsaa gjorde ; men da hun r kom til disse Ord : " Og paa lesum Christum " , taug > , hun stille , og blev gruelig plaget afDjcevelen . Hvore paa Luther sagde : " Jeg kjender dig Djcevel vel , du z saae gjerne , at jeg med store Ceremonier vilde besvcerge , dig , men jeg vil ei gjsre det " . Dagen derefter blev , hun bragt i Kirken , da Luther prcedikede , og efter Prcei diken l Sacrestiet , hvor hun faldt paa Jorden , siog i med Hcender og Fsdder ; dog reiste nogle Studenter e hende op. Derpaa erindrede Luther de Ncervcerende , at , den Magt at uddrive Djcevle , ei var mere i Kirken , den ' havde ogsaa , saasom den nu var plantet , den ei mere be hov ; desuden stulde man ei kere sig efter de pavelige Cere» monier , men bede imod denne <b>Aand</b> , og ei scette Gud <b>Maal</b> - eller Grcendser for , hvorledes og naar han stulde uddrive denne <b>Aand</b> , thi dermed blev Gud fristet ; men man stulde meget mere fare fort i Bsnnen , og med Taalmodighed vente den af Gud bestemte Hjcelpetime . Han lagde derpaa Haanden paa <b>Pigens</b> Hoved , sagde Fader-Vor og Troen , som ogsaa det Sprog Joh . 14 , 12. : " Hvo som troer paa mig , de Gjerninger , som jeg gjor , stal han og gjsre , og han stal gjsre stsrre end < disse . " Han bad iligemaade med de andre Kirke-Tjenere til Gud , at han for Christi Skyld vilde uddrive Djcevelen af denne <b>Pige</b> ; siden stsdte ban paa hende med Foden og sagde : " Du stolte Djcevel vilde gjerne , at jeg brugte store Ophcevelser imod dig , men seg vil ei gjsre det . Jeg veed , at dit Hoved er ssnderknusct , og at du ligger underkastet vor Herres Jesu Christi Fsdder." Derpaa gik han fra hende og man fsrte hende igjen til hendes Folk , som derefter strev , at hun ei mere blev Plaget afDjcevelen . Miniana skriver om Luthers Dsd : " Da Luther havde den Nat gjort sig formeget tilgode og taget til sig mere , end han " kunde taale , fandtes han dsd i Sengen . " Men mere redelig omgaaes dog Maimbourg , som med stsrre Sandhed beskriver hans Endeligt , hvor han kunde have faaet tilforladeligere Efterretning , end hans egen , som han ventelig af Ondstab selv har opdigtet . Da Churfyrsten fik afDr . Jonas Skrivelse Underretning om Luthers Dsd , strev han til Greverne Albrecht , Philip og Johan Georg til Mannsfcld , at han havde snstet , at Greverne havde ladet den gamle og udmattede Mand vcere uforvirret med deres besvcerlige Handeler , og tillige bevidnede i samme Brev , at han havde med et hsitbcdrsvct og bekymret Gemyt , fornummet Luthers Dsd . Man lod derpaa det afsjcelede Legeme til Kl . 9 ligge paa Sengen , og mange Mennesker kom og beskuede det med Graad og bittre Taarer ; siden kloedte man det i et nyt hvidt schvabist Overtrcek og lagde Liget i Kammeret paa en Seng af Straa , indtil en Tintiste blev stsbt , hvorudi han blev lagt . Da han nu laae i Kisten , kom atter en stor Mcengde Mennesker af begge Kjsn for at see hans Lug . Den 18 de Februar stod " Liget i Herberget , som var Dr. Tracbstedts Huus , hvor tvende Malere , den ene fra Eisleben , imedens han endnu laae i Stuen paa Sengen , og den anden , Lucas Fortennagel fra Halle ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111400044'target=_>James, John Angell, 1878, unge Kvinde</a><h4>26</h4>Fremdeles er det for Familielivet , saavelsom for Samfundslivet , übeskrivelig skjent og velgjsrende , naar <b>Kvinden</b> ved en rigtig Ledelse er dannet og forberedt til at opfylde fit vigtige Kald fom <b>Hustru</b> for at kunne vere Hjemmets stsrste Prydelfe og den virksomme , milde <b>Aand</b> , fra hvilken det huslige Liv henter Trivfel og Lykke under den fande Kjerligheds , den oplyfte Forstands og den oprigtige Bsns Magt , fom Apostelen siger-  Hjertets skjulte Menneske i en sagtmodig og stille Nands « forkrenkelige Vesen , hvilket er meget kosteligt for Gud . " 1 Pet . 3 , I — s . Det er ogfaa ved en saaledes beskaffen <b>Dannelse</b> og Sjelsretning , at den unge <b>Kvinde</b> , saavel som Mennesket i Almindelig ^ hed , ikke blot for sin egen Del bliver allerlykkeligst i Fedrehjemmet og under Udsvelsen af de forfljellige Kald i det almindelige Liv , men ogsaa faar Sind og Evne til at medvirke til andres Velferd og til at have en v. lgjsrende Indflydelfe paa mange . I alle Henseender giver denne Bog fortreffelig Undervisning og Veiledning . At en faadan kan tiltrcenges , var let at <b>vise</b> ved Paapegen af mange sorgelige , skjeve Retninger , der ere meget sedvanlige , iscrr i Henseende til den <b>kvindelige</b> Ungdoms Opdragelse , eller Mangel paa Opdragelse . og den urigtige Behandling , den faar , foruden mange Afueie , som ere at beklage .

<h4>81</h4>dens milde og retfcerdige Styrelse . Ved Kristendommen har <b>Kvinden</b> erholdt sin moralske og sociale Indflydelse , ja ncesten hele sin Tilværelse som et Samfundets Medlem . <b>Kvindens</b> Sjcel , hvilken mange af Fortidens Filosofer , Lovgivere og <b>Vise</b> ansaa for at vcere af ringere Vcerd og ude af Stand til at modtage <b>Dannelse</b> , er dog bleven udviklet under Kristendommens Beskyttelse . Kristi Evangelium, dette glade Befrielsens Budskab , er i denne forsomte Grube , hvilken endog de <b>Vise</b> have foragtet at oparbeide, og har derfra hentet op mangen uskatterlig Juvel , straalende af Snillets Ild og af den kristelige Kjcerligheds elskelige Farveglands . Kristendommen bar gjengivet <b>Kvinden</b> hendes tabte eller bortrovede Rettigheder og indsat de klareste Wdelstene i hendes Hcederskrone . Hendes tidligere Forfald forklarer i det mindste for en Del Übestandigheden af tidligere Tiders <b>Dannelse</b> . Det er umuligt for Samfundet vedvarende at holde sig oppe , naar dets <b>Kvinder</b> ere fornedrede til Slaver . Hvorsomhelst den <b>kvindelige</b> Slcegt ansees for underordnet , der bcerer Samfundet Oplosningens Spire i sit Skjsd og er bersvet Evnen til alt virkeligt Fremskridt . Det er umuligt , at Indretninger og Scedvaner , hvilke hindre eller kvcrle Naturens Instinkter og krcenke Guds Love , kunne krones med endelig Fremgang . Samfundet maa forandre sit ydre Udseende , maa fremstille Rigdommens Glands , Smagens Forfinelfe , Kunstens Forskjonnelse eller Videnstaoens og Literaturens mere vcerdifulde Fremstridt ; men hvis <b>Kvindens</b> Sjcrl forbliver uudviklet, hendes Smag udannet , hendes Perfon undertrykket , er Samfundets Grundvold usikker og vaklende , dets Sammenhceng svag . Hvor Kristendommen tjendes og forstaaes , der er ogsaa <b>Kvinden</b> fri . Evangeliet aabner kg en venlig Engel hendes Fcengsels Dore og indbyder hende til at gaa ud og nyde Sandhedens Solskin og indaaande den aandelige

<h4>104</h4>Jeg vil ogsaa omtale den lige Andel i Religionens Velsignelser , som skjenkes den kristne <b>Kvinde</b> , Hvor tydelig og bestemt har ikke Apostelen tilsikret ogsaa <b>Kvinden</b> Del i alle Kristi Velgjerninger , naar han siger :  Her er ikke Isde eller Grcrker ; her er ikke Tråel eller Fri ; her er ikke Mand eller <b>Kvinde</b> ; thi I ere alle en i Kristo Jesu . " Dette er et Fribrev , som skjenker <b>Kvinden</b> alle Frelsens Velsignelser; dette er Beviset for <b>Kvindens</b> levnlighed med Manden for Gud ; dette er <b>Kvindens</b> Rettighed til sin sande Plads i Samfundet . Der findes ingen Velsignelse , nodvendig for at vinde det evige Liv , hvilken ei i samme <b>Maal</b> tildeles <b>Kvinden</b> som Manden . Det er en almindelig Tro blandt Muhamedanerne , at <b>Kvinder</b> ikke faa Adgang til Paradiset ; Hourier * ) ere istedet scerlig stabte for denne Plads . Hvor fornedrende er ei en saadan Tanke ! Men den er i suldkommen Overensstemmelse med Muhamedanismens <b>Aand</b> , der betragter <b>Kvinden</b> mere som en Slave for Mandens Lidenstaber end som hans Ledsagerinde gjennem Livet , Kristendommen stiller <b>Kvinden</b> ved Mandens Side , som <b>Hustru</b> , som Datter , som Tjenestepige ; ved FamPearnen, i Templet , i Menigheden . Mand og <b>Kvinde</b> modes ved Korsets Fod og stulle ogsaa msdes i de Saliges Boliger. Kan vel noget saaledes som dette hceve og bevare

<h4>120</h4>det Baal , som fortcerer hendes afdsde Mand . Det har gjort en Ende paa <b>Kvindernes</b> Lidelser blandt de vilde Stammer , paa Indespcen ingens Rcedsier blandt Muhamedaner og Hedninger ; det har afskaffet Flergifte , Bornemord og Konkubinat i alle Lande , hvor det har havt Fremgang . la , Kristendommen er ncesten i alle Verdens Egne optraadt som <b>Kvindens</b> Befrier og Ven . Den har unddraget hende den ulykkelige Indflydelse af den muhamedanste Halvmaanes blege Skin og stillet hende i det oftlivende Lys af Retfcrrdighedens Sol . Den har frelst hende fra Krusefixets morke Magt og fort hende til Korsets hellige Hoider . Men ogsaa fra en anden Side kunne vi se , hvorledes Kristendommen selv blandt os har virket velgjsrende og forcedlende for Tusinder af engang ulykkelige og fornedrede <b>Kvinder</b> , ulykkelige ikke ved egen Skyld , men ved sine Mcends Grumhed og Laster . Hvor mange Hustruer have ikke levet iet Slags husligt Slaveri , foraarsaget ved Mandens Drukkenstab, Utrofiab og Tyranni , disse Mcend , som dog engang med Ed havde lovet at elfle og beskytte dem . Kristendommens Magt har i utallige Tilfcelde erobret og forandret saadanne Mcends Hjerter , lutret og helliget dem , saa at der senere syntes ikke at vcere mindre end en ny 3 Egtefcelle , end et nyt Menneske . Og et af de stsrste Beviser paa denne Forandring var hans forandrede Opfsrsel i Hjemmet, hvor Hustruen nu blev hans Selskab og Ledsagerinde istedetfor hans trcellende Slavinde . Saaledes har Kristendommen ved sin <b>Aand</b> og sine Forskrifter udvirket <b>Kvindens</b> Befrielse og Forcedling , hvorsomhelst den har vundet Indgang. Middelalderens Ridderlighed , der var en Forming af Religion , Tapverheo og Hsflighed , sogte , saa taabelig den end undertiden viste sig , dog i nogen Grad at naa dette <b>Maal</b> . Jeg vil ikke bestride Sandheden af den Bemcerkning, som er gjort af en fransk Forfatter , der siger .

<h4>282</h4>Til en <b>vis</b> Grad er det <b>Kvinden</b> , som opretholder sit Folks Lykke . Hendes Dyd staar , hvis den er fast og ufordervet, som en beskyttende Vagt over Folket . Lov . Retfcerdighed og Frihed , Kunst og Videnskab , alt bidrager naturligvis til et Folks Velgaaende , men hvis <b>Kvindens</b> almmdelige Scedelighed staar lavt , saa er alt andet forgjceves, medens derimod , hvis <b>kvindelig</b> Dyd og <b>Dannelse</b> er almindelig , Civilisationens Strsm stiger til sin storste Hside og bliver frugtbringende for den mest udstrakte Overstade .

<h4>353</h4>sundt Omdomme og rigtig Takt , det ansees for mindre vigtigt , medens Viljens Retning og Hjertets Foredling paa en saa bedrsvelig Maade forssmmes . Jeg arbeider ikke paa Opofrelse af noget , som kan stjcenke <b>Kvindens</b> <b>Sjel</b> foroget Ynde ; jeg onffer blot , at Bisager ikke stulle betragtes som Hovedsager ; jeg vil have <b>Visdom</b> tilligemed Ynde , virkelig <b>Kundskab</b> og ikke blot en lysende Overfladiskhed . Jeg vil se <b>Kvinden</b> opdraget til Mandens Selskab , ikke til hans Legetoi . Jeg snster at se hende prydet med noget hsiere og bedre end blot modernt Kram , tom Effekt og affekteret Vcesen ; jeg onster at se hende udrustet med Evne til al beholde sin Wgtefelles Hjerte ved at lere ham at scette Pris paa hendes Omdsmme ; jeg vil se hende indgyde sine Bsrn Agtelse og Tillid og i det Hjem , hvor hendes Indflydelse er saa stor . opdrage Mcend og <b>Kvinder</b> , som kunne forsge Folkets 3 Ere og Magt . Maatte Msdre herved bistaaes af dem . hvilke de betro sine Dotres Opdragelse . Det er ssrgeligt at tenke paa , at en stor Del af Msdrene endnu ere ude af Stand til selv at opdrage dem ; endnu ssrgeligere er det at betenke deres Udnelighed , til hvilke denne Dpdragelse sedvanlig er overgivet . Hvor lidet have de betenkt Vigtigheden af sit Kald ! Hvor faa have gjort sig Rede for <b>Kvindens</b> Opgave i Verden , have overveiet , hvad som udfordres til at udstyre hende derfor , og hvorledes de maatte lunne hjelpe hende til at opnaa dette Udstyr . Hvor sjelden betenke de sin Pligt paa al mulig Maade at ssge at meddele ikke blot Kundfkab , men ogsaa <b>Visdom</b> , ikke blot at uddanne mere eller mindre mislykkede Kunstnerinder , men om muligt at forberede Marken for Kristendommens helligende Virkninger , at opdrage Barnet til en forstandig <b>Kvinde</b> , en god Mgtefelle , en viis <b>Moder</b> , et nyttigt Medlem af Samfundet .

<h4>452</h4>Moder gjor Fordring paa til Bistand i Husholdningen , er scedvanlig ganske eders egen . Eders Brsdre , de maa nu vcere hjemme eller borte , maa nsdvendigvis vcere mere sysselsatte . Deres Tid er i de fcerreste Tilfcelde deres egen , noget , som man altfor ofte fremhcever og endog anfsrer som Undstyldning for Forssmmelighed i , hvad som angaar Religionens Fordringer og Sjcelens Frelse . I have ingen saadan Undskyldning . Eders Tid er saa meget eders egen , at I kunne gaa og komme , lcese eller arbeide ester eget Behag . I have endog saa megen Frihed , at Tiden undertiden synes eder altfor lang , og jeg frygter for , at nogle af eder maaste forsde nogle Timer hver Dag paa unyttigt Arbeide uden <b>Formaal</b> . I have mindft af alle Grund til at klage over Mangel paa Tid til at tamte paa Gvigheden eller Mangel paa Leilighet ) til at ssge eders Salighed . Er det muligt , at I kunne vcere blinde for eders ncervcerende Fortrin , at vcere fri for ncesten al Slags Bekymring ? I ville maafle engang sande Apostelens Ord :  Der er Forstjel imellem Hustruen og Jomfruen . Den ugifte har Omhyggelighed for de Ting , som hore Herren til , at hun kan vcere hellig baade paa Legeme og i <b>Aand</b> , men den gifte har Omhyggelighed for det , som hsrer Verden til , hvorledes hun kan behage Manden!" Hvor megen <b>Lcerdom</b> , Advarsel og Raad ligger ikke i disse faa Ord ! Apostelen vil ikke sige , at alle ugifte <b>Kvinder</b> handle saa , og vi vide desvcrrre , at altfor mange af dem ikke sporge efter at behage Herren , men hans Mening er , at den ugifte , fri for en Wgtefcelles , Moders og Husmoders Bekymringer , har mest Anledning til at tcenke paa Sjcelens Anliggender . Ak , unge <b>Kvinde</b> , du kan muligvis se , hvad der forestaar dig , hvis du betragter , hvad der er blevet din Moders Hoved , Hjerte og Hcender tildel .

<h4>499</h4>Og nu ved Afflutningen af dette Kapitel , lader mig opfordre eder , I unge , til allerede ved Livets Begyndelfe at betcenke det store <b>Maal</b> , for hvilket I af Skaberen fattes ind i Verden . Tcrnker ikke altfor hoit om eder selv , thi I ere Syndere , I saavelsom alle . I vehsve , I ogsaa , og I kunne ogsaa vinde den Frelse , som er beredt i Jesus Kristus . Tamker ikke heller altfor ringe om eder ; thi I ere udsdelige Vcesener og kunne arve et evigt Liv , Hcever eder til eders fande Vcerdighed ved at hoeve eder til en sand Gudsfrygt , Tro og et Liv i Sandheden . Forspilder ikke eders Liv i smaaligt , om ogsaa uflyldigt Tidsfordriv , uvcerdigt eders Evner , eders Bestemmelfe og eders Pligt . Betragter med klart Blik og fuld <b>Bevidsthed</b> eders Kald for denne Verden og eders Bestemmelse for den tilkommende . Ser , hvorledes et Kald her aabner sig for eder , hvilket I maa opfylde med en Kristens Hellighet » , Mre og Velsignelse saavelsom med alle en <b>Kvindes</b> Dyder og Fortrin . Drager eders Hjerte fra Forfcengeligheden, stjcrnker det til Gudsfrygten . Skjcenker Gud eders Livs Morgen , og da stal dets Dag , den blive lang eller kort , klar eller overskyet , ensom eller tilbragt i Forening med

<h4>501</h4>andre , lys eller msrk , rolig eller stormende , om den end ender ved en hastig Hjemssgelse , saadan som eders Kjsn vel kjender , eller om den gaar ned i en lang og mork Aften , den skal dog for eders salige <b>Aand</b> , befriet fra Stsvets Byrde , aflsses af den skyfri og evige Morgen , som ikke efterfslges af nogen Nat . Da stulle I finoe , at <b>Kvindens</b> saavelsom Mandens <b>Maal</b> var at forherlige Gud og nyde af hans Vcesen for evig .

<h4>514</h4>I denne Henseende vilde jeg endog opvackke eder tll Nidljcrrhed . Apostelen siger :  Det er godt altid at vwre nidkjcrr i det gode " . Nidtjcerhed betyder , som I vide , en inderlig Attraa , som virker en tilsvarende Kraft til at handle i en <b>vis</b> Hensigt og sor et eftertragtet <b>Formaal</b> , og naar denne Nidkjcerhed er rettet mod et rigtigt <b>Maal</b> , betegner den en cedel og ophsiet Sindsstemning . Tette Sindelag fordrer imidlertid stor Forsigtighed i sin Virksomhed , iscrr hos de unge og fremfor alt hos <b>Kvinden</b> . Det ligner Ilden, som visselig kan benyttes til mange nyttige <b>Formaal</b> , naar den anvendes ret og passes , men hvis den forlades

<h4>527</h4>alle Sind ere her optagne , alle Hcender ere i Virksomhed, Mcends , <b>Kvinders</b> og Vorns . Maatte vi drage <b>Lcerdom</b> af dette stette Exempel , og maatte vi <b>vise</b> samme Nidkjcerhed for den sande Gud , som de frafaldne Isder for de falste ! Kristendommen . kan ligesaavel som Hedenskabet afgive en Mark for <b>Kvinder</b> og Born til at bearbeide ; og hvor hellige ere ikke de Forpligtelser , som den paalcegger sine Bekjendere om at udbrede den .

<h4>554</h4>staber have uden Tvivl undertiden vceret Felgen , og det er derfor vigtigt for alle , som deltage i denne Beskjeftigelse , at vcere saa forstgtige og aarvaagne , at ikke deres Opforsel maa give dem , som soge derefter , Anledning til at tale ilde om Ssndagsstole-Undervisningens Indvirkning paa deres Karakter og Opfsrsel , som give sig af dermed . Distriktsforeninger og andre Kjcerlighedsindretninger for Afhjcelpelse af <b>aandelig</b> og legemlig Nsd under Tilsyn og Ledelse af <b>Kvinder</b> ere blevne ganske almindelige blandt Kristne af forskjellige kirkelige Samfund . Det var vel at snfle , at disse saa lidt som muligt lededes as yngre Fruentimmer. Den modnere Alders Erfarenhed og Anseelse behsves for at give saadanne Arbeider dens rette Kraft og Virkning , hvilket dog ikke hindrer , at ogsaa de yngre <b>Kvinder</b> kunne deltage i disse gode Gjerninger ved de celdres Side . ' Bibel- og Missionsselstaber samt andre fromme Stiftelser have ogsaa aabnet en Mark for <b>kvindelig</b> Virksomhet » . Mange <b>kvindelige</b> Hcender ere sysselsatte med at indsamle og skaffe tilveie Midler for disse vigtige <b>Formaal</b> og forsyne de Fattige med Guds Ord . Herimod kan intet indvendes, blot det iagttages , at ikke altfor unge Personer anvendes derved . Man bsr herved paase , at intet ster , som

<h4>570</h4>Kvindens Anlceg for en flyvende og let Stil er almindelig erkjendt ; burde ikke ogsaa denne Gave helliges til Guds Tjeneste og Medmenneskers Gavn ? Hvor mange ere ikke derved blevne ledede til <b>Kundskab</b> om Gud , til Jesu Tro ! Der ligger en stor Magt i et vel strevet kristeligt Brev . Man kjender og fornemmer , at her er en Udstrsmning af Kjcerlighed fra Hjerte til Hjerte . Man lceser det i Ensomhet), naar ingen kan vcere Vidne til dets Virkning , man forstyrres ilke engang af Forfatterens bebreidende Ncervcerelse. Man bliver ikke gjerne foruroliget , man tåger ikke gjerne den fravoerende Vens Ord ilde op. Unge <b>Kvinder</b> , anvender eders Penne , og indgyder paa denne Maade en Kjcerlighedens <b>Aand</b> i eders Brev !

<h4>587</h4>Kjcerlighed til Virksomhed er et Gode ; Lidenstab derfor er derimod et Onde . Der gives en forstandig , Edruelig og bibelsk Nidkjcerhed ; der kan ogsaa gives en Nidkjcerhed, som blot er Kjcerlighed til Adspredelser under Religionens Skin . Naar en ung <b>Kvinde</b> elsker Hjemmet og dets Pligter , men er villig til ved passende Leilighed og for et vcerdigt <b>Formaal</b> at deltage i den ydre religisse Virksomhed og almindelige Velgjsrenhed , da har hun det rette Sind . Men hvis Hjemmet med dets Pligter forekommer hende kjedeligt , og hun stedse og altid lcenges efter ydre Virksomhed , da er allerede hendes Smag fordcervet , hendes Karakter skadet , og hendes fremtidige Udsigter til en <b>vis</b> Grad fordunklede . Hvis hun har forsaget den Tanke eller det Dnfle nogensinde at blive en gift <b>Kvinde</b> , og istedet , uden at tcenke paa at gaa i Kloster , dog har besluttet at blive en Barmhjertighedssoster , da kan det visselig vcere i sin Orden ganske at hengive sig til velgjsrende Virksomhed ; men hvis dette ikke er Tilfceldet , maa hun tåge sig ivare for de Vaner , som maaske engang ville

<h4>640</h4>under Sagtmodighedens Slor , men den bor dog <b>vise</b> sig , ikke dslges af det lyse Hylle , og naar din Bekymring blot leilighedsvis drister sig til at tale , saa gjsre du det blot i Kjoerlighed . En saadan Opfsrsel kan naa sit <b>Maal</b> og frembringe velsignelsesrige Virkninger . Da kan Folgen blive den samme som i de smaa Fortellinger , jeg tåger mig den Frihed at anfsre til Belysning heraf .  En <b>Kvinde</b> , som lenge havde vcrret kjendt og agtet for sit stille , kristelige Vesen , gjorde et Besog hos sin Prest for at forestille for ham sin bedagede <b>Moder</b> , der ststtede sig paa hendes Arm og syntes at forlade sig paa hende med den Tillit » , som er saa behagelig og vederkvegende for den bedagede <b>Moder</b> og saa hjertegledende for et omt og taknemligt Barn . Begge vare overvceldede af Fslelser , og nogle Dieblikke forlov , uden at nogen sagde noget . Lcereren bad dem sidde ned og begyndte :  Ruvel , Hanna , jeg formoder , det er din kjcere <b>Moder</b> ; jeg er glad ved at se hende ! "  Ia " , svarede Moderen med afbrudte Ord ,  hendes <b>Moder</b> og Datter tillige ; for fem og tyve Aar siden fsdte jeg hende til Verden , og nu haaber jeg , at jeg ved hende er bleven fodt paany for Gud . " " — En anden lignende Beretning . „ , M , " sagde en <b>Moder</b> gredende , da hendes Datter havde anmeldt sig til Deltagelse i Herrens Nadvere ,  jeg kan ikke lcenger modstaa . Hvor kan jeg se mit tjcrre Barn elske og lcese Skriften , medens jeg aldrig aabner min Bibel , og hvorledes kan jeg se hende soge Herren , medens jeg derimod aldrig beder ; hun gaar til Herrens Bord , men for mig er hans Dsd af intet Verd ! Jeg ser nu " , sagde hun under Taarer til Presten , som bessgte hende for at underrette hende om hendes Datters Regjering ,  jeg ser nu , hun har Ret , men jeg har Uret ; jeg har seet hende sterk under Forsmcedelsen , taalmodig ved Bebreidelsen , glad og tilfreds under alle sine Lidelser . Da hun sidst var syg . og hun

<h4>714</h4>Det er stjsnt at se en ung <b>Kvinde</b> ikke alene underholde sig selv , men ogsaa ved Udbyttet as sin Flid og sine Besparelser bidrage til sine bedagede , svage og maaske fattige Forceldres Velbefindende . Mangen saadan <b>Heltinde</b> har forladt Hjemmet og for dette <b>Formaal</b> underkastet sig Forsagelser , som hverken have vcere faa eller übetydelige . Alle Vanskeligheder bar hun med Taalmodighed ved den Tanke , at hun derved lykkeliggjorde sine elskede Forceldres Hjem . Her behoves dog en Advarsel mod en altfor stor Iver for at komme bort fra Hjemmet . Gn stor Familie i trcrngende Omstcendigheder medfsrer megen Forsagelse og Bekymring ilke blot for Moderen , men ogsaa for de cedle Dstre . I et saadant Tilfcrlde , hvor en ung <b>Kvinde</b> kan verre til vcrsentlig Hjcelp for sin <b>Moder</b> , der snfker at beholde hende hjemme , bsr Datteren ikke snste at drage bort og overlade sin <b>Moder</b> til ensom at strcrve under den Byrde , som trykker og maaske knuser hende ; thi dette vilde <b>vise</b> en beklagelig Mangel paa barnlig Dmhed . Det er stjsnt , det er lsftende at hore en Datter afgive denne Bekjendelse :  Hvorsomhelst , under hvilkesomhelst Omstcendigheder , i eller udenfor Hjemmet , gift eller ugift , er jeg beredt og villig til at opofre mig for eder , mine elskede Forceldre . "

<h4>751</h4>dine Pligter . De ere blevne overladte til din Varetcegt og se op til dig som sin <b>Lærerinde</b> og sit Gxemvel . Hvis du vil gjsre dit sidste Regnskab for Gud med Glcede og ikke med Sorg , gjor da dit bedste ; anvend alle dine Krcefter og Evner paa at uddanne deres Karakter , udruste deres Forstand med Kuudskaber og paa enhver Maade berede dem for den Plads , som de ere bestemte til at indtage i Livet . Bered dig selv til dette Arbeide ved fortsat Lcrsning og Selvstudium , vcer ikke tilfredsstillet med den Dygtighed , du allerede besidder . Gt fornuftigt og udodeligt Vesens Opdragelse for denne og den tilkommende Verden er et vigtigt Verk . Beflit dig selv paa <b>aandelig</b> Udvikling for at kunne befordre andres . Ssg at vinde deres Tiltro ved din Duelighed , deres Hengivenhed ved din Venlighed , deres Agtelse ved din Verdighed og deres Lervillighed ved din egen Flid . Lad dit <b>Maal</b> vere at meddele ikke blot <b>Kundskaber</b> , men ogsaa <b>Visdom</b> , ikke blot at belesse Hukommelsen, men ogsaa at ove Dsmmekraften , styrke Viljen og vekke Samvittigheden , at lere dem ikke blot at tcenke rigtig , men ogsaa at handle ret og samvittighedsfuldt , i hvilken Livsstilling de end komme til at indtrcede .

<h4>852</h4>Merk her , af hvilken Vigtighed det er i Tide at undfly stette Vaner , fordi Gfterladenhed og Udsvelsen deraf saaledes indforlive dem med vort eget Vcrsen , at de med Tiden blive vor anden Natur og styrkes af Alderen istedetfor at udslettes . Vi finde hos Isak de tyende Hovedfeil , som lige indtil Alderdommen beherskede ham , at vcere hans Tilfredsstillelse af sin Begjcrrlighed efter en <b>vis</b> Slags Mad og hans Partisthed for sin Son Gsau . Rebekka , hvis Hengivenhed altid var vaagen , nidkjcrr og virksom for sin Yndling , hsrte den Opfordring , som hendes Mand gav Esau , og strax udkastede hun en Plan til at vende Velsignelsen til den anden og , som hun vidste , udvalgte Ssn . Hvad havde vceret hendes Pligt ? At gjsre Isak sagtmodige Forestillinger om det utilbmlige i at handle i fuldkommen Strid med Guds aabenbarede Raad og Beslutning . En saadan Paamindelse til et sagtmodigt og gudfrygtigt Sind , som hans utvivlsomt var trods sin Svaghed , vilde efter al Sandsynlighed have havt god Fremgang . Istedetfor dette lagde hun for Dagen , hvad der altid er bleven anseet for at hsre til <b>Kvindens</b> Svagheder, en Tilboielighed til at tåge sin Tilflugt til List , Kunster og Kneb , et Dnske efter heller at naa sit <b>Maal</b> paa krogede Omveie ' end med aabent og rent Spil . Det er overftsdigt at gaa md paa Enkelthederne i hendes Plan , den hurtige Udforelse og dens Held . Fortellingen er ssrgelig . Intet andet end Svig og Bedrageri , uvcrrdige

<h4>872</h4>for alt stjsnt og yndigt . lal sin Storhed og Mangfoldighed synes dog dens Gang saa utvungen og let , fordi det hele ledes og styres af en <b>Visdom</b> og Magt , som virker ikke blot uden nogen synlig Anstrengelse , men endog under en fuldkommen Hvile . Det naturlige er ikke mindre skjont i Kunsten end i Naturen , og Kunstens Fuldendelse er just det selv at forsvinde , idet den afbilder det naturlige i Naturen , Dette gjcelder ogsaa om Adfcerd og Ovfsrsel ; der iscer er Affektation afstyelig og moobydelig . En ssgt og tilkunstlet Adfcerd , vcere sig , at det gjcelder <b>Kundskaber</b> eller Dyd , Konversation eller blot endog Udtale , Klceder eller Sedvaner , er altid vederstyggelig . Den kan ei vinde nogen Agtelse eller Beundring , den maa kun vcekke Spot . Maaske dette er en af de fornemste Daarstaber , mod hvilke <b>Kvinder</b> , og iscer unge <b>Kvinder</b> af Opdragelse , bsr vcere paa sin Post . Gt affekteret Vcesen vil sige meget ; det indeholder nemlig noget usedeligt og vanhelligt , det hentyder paa List , Tvetydighet , og forstilte Dyder , som ikke findes . I sin bestemte Form er det Hykleri , den afskyeligste Last paa Jorden . Nu har jeg dog egentlig for Vie en affekteret Adfcerd ; et hyklerisk Skin af Overlegenhed i et eller andet , en tillcert Rolle for at lyse eller blcende . Dette udspringer sedvanlig fra den Forfccngelighed , som til alle Tider er bleven anseet for at vcere en af Kvindekjonnets Svagheder , og hvis Tilstedevcerelse forstyrrer mangen ellers elstvcerdig Personlighed . Forfomgeligheden er for <b>Kvinden</b> , hvad Mrgjerrigheden er for Manden , og stjsnt den kan vcere mindre farlig , er den dog en smudsigere og hcesligere Feil ; det er en Form af Egenkjcerlighed , ligesaa avindsyg som umcettelig Intet kan vcere mere stridende mod Evangeliets <b>Aand</b> , og den eneste Beskyttelse derimod er en dyb Hjertets Idmyghed . Den maa I ifore eder , I unge ; det er den

<h4>885</h4>I et foregaaende Kapitel antydede jeg , at den overdrevne Vlufcrrdighed , som ei engang drister sig til at tale med det andet Kjsn , ei altid er af det reneste Slags . Der staar endnu tilbage nogle Lcrrdomme , som en <b>Kvinde</b> har at hente af Rebekkas Opfmsel under sit senere Liv , Lcrrdomme at medbringe og ihukomme under hele Livet . Den fsrste er en almindeligere : forandrede Omstomdigheder medfsre ofte forandret Karakter og forandret Opforsel . Hvor forskjellig var ikke her Jomfruen i Mesopotamien og <b>Husmoderen</b> i Kana ' an ! Og er det vel noget usedvanligt ogsaa nu at se Forandringer , langt mere omfattende og gjennemgribende , fslge paa en <b>Kvindes</b> Indtrcedelse i Wgtestanden? Betragt videre denne enkelte <b>Lcerdom</b> , at vi aldrig bsr arbeide for et. godt <b>Formaal</b> med slette Midler , eller med andre Ord , ikke gjsre det onde forat deraf maa fslge det gode . Assty denne lesuiternes Grundscetning og Dndlingssats, at  Hensigten helliger Midlet , " og affly i Særdeleshet, at fslge denne ugudelige Grundscetning i det , som angaar Religionen . Hvilke Forbrydelser ere ei i Religionens misbrugte Navn blevne udsvede af romerste Ivrere for deres Kirkes Nedste ! Og ogsaa af andre , i Navnet Kristne , med renere Tro og besjcrleoe af en mildere <b>Aand</b> , hvor me- ' get er ei bleven gjort , som det hed , for Religionen , men i Virkeligheden kun for at befordre scrregne Hensigter , i Strid med alt , som Guds Lov og Kjcerlighed til Nersten byder . Religionen vil ei betjenes af nogen anden <b>Aand</b> og bevcrgende Kraft end sin egen ; den foragter alle Tilbud , som stride mod Sandhed , Kjcrrlighed , Redelighed og 3 Ere . Og jo stsrre vor Nidkjcerhed for en Sag er , desto mere ere vi i Fare for ved Vanskeligheder eller Modgang at tåge vor Tilflugt til uvcerdige Midler ; saaledes , jo mere nidkjcere vi ere i at ssge at gjsre noget for Guds Sags Fremme , desto mere bsr vi verre paa vor Post , at vi ei

<h4>907</h4>Omsorg at pryde :  Det stal vcere , " siger han ,  Hjertets skjulte Menneske i en sagtmodig og stille Aands uforkrcenkelige Vcrsen , hvilket er meget kosteligt for Gud . " Hvor skjon er dog ikke denne Fremstilling , hvilket Dyb as Tanker ligger ei i disse faa Ord ! Det er Sjcelens og ikke Legemets Prydelse , som iscer bor lcegge Beslag paa den kristne <b>Kvindes</b> Opmersomhed . Hvilke ere de Prydelser , som anstaa Sjcelen ? Sjcelen er udsdelig og uforgjcengelig; saaledes bsr ogsaa dens Prydelser vcere . Hvorledes kunne Gnld og Juveler passe dertil ? Kan vel Diamanten oplyse eller overstraale Fornuftens Lys ? Eller Rubiner forhsie Hjertets Ande ? Eller den cegte Perle indfattes i Samvittigheden ? Eller en straalende Dragt klcede Karakteren ? Eller Fjcer og Blomster vaie over en ny og himmelsk Natur ? Nei , Sjcelens Prydelser ere Sandhed , Hellighet » , Tro , Haab , Kjcerlighed , Idmyghed , <b>Visdom</b> , Glcrde , Sagtmodighed og Mildhed ; disse ere de Juveler , hvormed det indvortes Menneske burde pryde sig . Det udvortes , det jordiske er forgjcengeligt . Stov er det , Stsv stal det atter blive . Hin Skjsnhed , som nu straaler i Juveler og Mdelstene , er Prydelse for Valsalen og en Gjenstand for Avind , vil inden kort vcere en Masse af Forraadnelse , altfor gruopvcekkende til at beskues , siden et vederstyggelig ! Skelet , en Samling af Benknogler , en Haandfuld Stsv , og hvor er da Sjcelen — den udsdelige <b>Aand</b> ? Mon den stal bcere Legemets aflagte Pynt ? O nej , de have maaske endnu undergaaet Forgjcengeligheden , dog blot for at pryde andre til Dsden hjemfaldne Legemer , men ei til at berede Sjcele for evig Herlighed . Men tcenk nu paa den <b>Kvinde</b> , som , uden at lcegge ' altfor stor Vcegt paa Legemets Udstyrelse , fsrst og fremst efterstrcebte Sjcelens Prydelser ; se hende , som var ifsrt Retfcerdighedens Dragt og Salighedes Klcedebon , prydet med en ydmyg og stille

<h4>945</h4>stelen taler , naar han benytter det Udtryk ,  at de kunne pryde Guds , vor Frelsers <b>Lcerdom</b> i alle Ting ! " I alle Menigheder findes mange af den tjenende Klasse , og jeg er glad ved at sige , at <b>kvindelige</b> Tjenere ofte ved sin med sin Bekjendelse overensstemmende Vandel er en virkelig Prydelse for sin Menighed og bidrage meget til at indgive sine Herrer Tiltro til Sandheden . Ogsaa have iscer de fremfor andre rig og gunstig Anledning til med Idmyghed , Mildhed og Taalmodighed at pryde Kristi Lcere . Paa den anden Side er det ssrgeligt , at mange , som have havt <b>aandelig</b> stndede Tjenere i sit Hus , senere have havt Grund til at sige , at de aldrig mere snste sig dem , thi de ere i enkelte Tilfcelde ei bedre , og i nogle blot vcerre end andre .

<h4>953</h4>En <b>vis</b> Smag i , hvad man sedvanlig kalder Talenter eller <b>Dannelse</b> , er prydende for Religionen , naar den ei drives til Merlighed . Jeg har allerede forhen bemerket, at saadanne Ting , som hsr ^ r til Kundstab og Dannelse, ei ere at foragte . Vel indrommes , at det er en Synd at optage Time efter Time hver dyrebar Dag af Livet blot med de moderne tidsforsdende Beskjeftigelser , Broderier , Kunstvcevning , Maling , Sprog , Musik m , m . , medens Religionen , nyttige Bsger og Kjcerlighedsgjerninger forssmmes . Det er i Sandhed haardt at se et fornuftigt, udsdeligt Vcesen , som lever i denne Vankundighedens , Syndens og Elendighedens Verden , naar hun kunde- gjsre noget til dens Oplysning , Forbedring og Velsignelse , istedet optage den vigtigste og bedste Del af sin Tid med at udarbeide Msnstre paa Stoffe eller male dem paa Papir eller lcere Sprog og Musik . Lad mig dog ikke nu misforstaaes . Jeg er ikke en saa stcerk Forsvarer af blot det nyttige , at jeg ikke ogsaa elsker at se det stjsnne og smukke virkeliggjsres . Jeg snsker vist ikke , at <b>Pigerne</b> bortlcegge Naalen og Penselen , lade Instrumentet urort og Sprogene ulcerte . Nei ingenlunde ! Religionen forbyder intet saadant . Natur og Bibel ere jo opfyldte med det stjonneste Broderi , af henrivende Musik og fortryllende Farver og tale i mangfoldige Dialekter Sprog , som ere os fremmede . Istedetfor at forbyde , hvad der kan forsge Kvindens

<h4>1090</h4>Msdre burde derfor vere vel fortrolige med det Arbeide, som er dem paalagt . Jeg tcenker her ikke paa Bornenes Legeme , og hvad det tiltrenger , men deres Ovdragelse i moralsk og religios Henseende . En <b>Moder</b> tilhorer det at uddanne Karakteren ; hun har at meddele ikke blot Kundstaber , men ogsaa Vaner . Hendes seregne Opgaver er at indvirke paa Hjertet , at vaage over Vaner og Tilboieligheder . Hun bor betragte sine Born som vordende Medlemmer af Samfundet , Familiefedre eller <b>Husmodre</b> , men fremfor alt som de , der her stulle opdrages til Medborgere i Himmelen . Hun maa , jeg gjentager det atter , fornemmelig opstille det som sit <b>Maal</b> at uddanne sine Boms Karakter for begge disse Verdener . Lcerere ville sandsynligvis fremdeles anvendes til den videnskabelige Undervisning , men Moderens Kald er fra den forste Begyndelse at uddanne Forstanden og Vanerne . Mange have ingen anden Tanke om Opdragelsen , end at den er Meddelelsen af Kundstaber . Meget er i senere Tider blevet sagt om Forskjellen mellem Opdragelse og Undervisning . De have virkelig forskjellig Betydning . At undervise er at underrette om , gjore en klog paa eller bekjendt med noget ,  indlegge " Kundstaber i <b>Sjelen</b> , som  instruere " efter Ordlyden betyder . At opdrage derimod eller educere" er at fremdrage eller fremkalde til Virksomhed <b>Sjelens</b> egne Livskrefter , at vekke og styrke dens Tanke og Vilje . Begge Dele er en <b>Moder</b> ansvarlig for , fordi det er hende , som stal uddanne Karakteren . Begreber maa meddeles Barnet , og det maa undervises om , hvorledes

<h4>1133</h4>Og dog , tidligere end dette kan ste , er Plads for Moderens Indflydelse . En af de storste Feiltagelfer , som dog er saa sedvanlig blandt Msdre , er , at de fsrste to eller tre Aar af Barnets Liv ei ere af nogen Betydning for dets Opdragelse . Og dog er det sande Forhold det , at de ere de allervigtigste . As de Indtryk , som modtages , de Grundsandheder , som indskjoerpes . og de Vaner , som erhverves i de Aar , kan Barnets Karakter bestemmes baade for Tid og Gvighed . Og klart er det , at Barnet har sedelig <b>Bevidsthed</b> , forinden det kan tale . Samvittigheden eller Evnen til at gjsre Forstjel mellem godt og ondt bor ved en viis <b>Moder</b> kunne udvikles hos Barnet strax efter , om ei fsr dets anden Fsdselsdag . Allerede da kan gjores begribeligt for det , hvad " dets <b>Moder</b> anser for ret eller uret , hvad der behager eller ikke behager hende . Vi hsre Msdre ofte sige , at deres Bsrn ere endnu for unge til at lcere at adlyde . Men den <b>Moder</b> , som fslger den Grundscrtning , at Barnet stal kunne raade sig selv nogle Aar eller idetmindste nogle Maaneder , vil til sin egen Skade finde , at den <b>Lcerdom</b> idetmindste ei saa let tages af Barnet . Moralsk eller sedelig Opdragelse kan og bor gaa foran den , som blot skjcenkes Forstanden . Med Hjertet

<h4>1143</h4>af at dine Bsrn ere bestemte for en Evighed og stabte med Evne til at nyde Himmelens Salighed , bor du allerede fra Barndommen af ssge at vcekke hos dem Hu og Sind for det aandelige og evige . Det er Evigheden , som med sin Ncervoerelfe gjsr det lille og übetydelige stort og vigtigt, og det er ogsaa deraf , at en Moders ophsiede Vcerdighed ien ikke ringe Grad er betinget . Hun har paa den Almcegtiges Bud givet Livet , ikke til et Vcesen , forn stal ds og forgaa lig Dyrene paa Marken , men til et udsdeligt ! Hendes diende Barn , svagt og hjcelpewst , som det nu ser ud . bcrrer dog i sig en fornuftig Sjcel , Evne til at fsle og tcenke , et Liv , forn ingen Tid kan forstyrre . Himle og Jord stulle forgaa med sin Pragt , men det fattigste, det ringeste , det usleste Barn har en <b>Aand</b> , som stal leve i alle Evigheder . <b>Moder</b> , dette er din Vcerdighed . Fol og vurder den Udmerkelse , dlg sier , naar du stal kaldes til at opdrage den almcegtige Herres Ssnner og Dstre og at forberede Medlemmerne af den hellige Familie , forn stal bo i de mange Boliger i Faderhufet , hvor lefus er gaaet hen for at berede dem Sted . Hengiv dig med Hjerte og Sjcel , med alle dine Evner til dette cerefuldeDcerk . Men vcer i alt forsigtig og betcenkfom , faa at du ei fnarere opvcekker Uvilje og Affly end Lyst og Tilbsielighed for den sande Gudsfrygt . Ifsr d ' ig Kjcerlighed , Mildhet , og Glcede , naar du underviser dit Barn i selve <b>Visdommens</b> Begyndelfe. Lev og vcer for dem just saaledes , som du lcerer , at Herren byder . Lad dem lcere af din Karakter og Handlemaade ligefaa vel forn af dine Ord . Og vcer ifcer omhyggelig i ikke at fremholde forn et Lovbud det , forn blot burde vcere en Kilde til Glcede , en Gjenstand for Haab . Lad dine Bsrn se paa dig , at om Gudsfrygtens Vei paa den ene Side er trang og smal , saa er den dog paa den anden Side  en herlig Vei og alle dens Stier Fred . " Tving ei

<h4>1148</h4>Paa den ostlige Kyst af Long Island , i en af de mere ensomme Egne i Amerika , levede for nogle Aartier siden en rigt begavet <b>Moder</b> , som engang i sine Optegnelser strev fslgende simple Ord :  Imorges stod jeg tidlig op for at bede for mine Born og iser , at mine Ssnner maatte blive Ordets Tjenere og virke som Missionerer for . lefus Kristus . " Nogle Aar fenere giver en ncervcerende Ven fslgende Skildring af denne Moders sidste Dieblikke :  Paa Grund af yderlig Mathed var hendes <b>Bevidsthed</b> ikke fuldkommen klar , hendes Ord usammenhengende ; men da hun traadte ind i Dodsskyggens Dal , blev hendes Sjcel lys og Morket forfvandt . Hun opsendte en brcendende Bsn og fortalte siden for sin <b>Moder</b> , at hvad hun derunder fslte og erfarede , var saa megtigt , at hun neppe kunde udholde det , og at et sterre <b>Maal</b> havde veret hendes Krefter for

<h4>1213</h4>til Letsindighed . Intet er mere utilladeligt end en tanlelss og verdslig <b>Moder</b> , som tillader upassende Tale eller tomt Sladder eller lsst Snak at foregaa i sin Ncervcerelse mellem Bsrnene , At kaldes en  Sladderssster " er ikke anbefalende for nogen , allermindst for en <b>Moder</b> . Og af al Skjemt , som en forstandig og omhyggelig <b>Moder</b> stal undvige, ere Forelskelser og Kjcerlighedseveutyr det , som der mindst bsr vcere Tale om For sine Ssnner bsr hun vcere det Msnster , ester hvilket hun vilde se dem vcelge sig Hu struer , for sine Dstre det Gxempel , hun onster , de vilde sslg ? , hvis de nogensinde selv blive Hustruer og Msdre . Hendes Holdning bor vcere hsi og vcerdig , men formildet ved en Kjcerlighedens Aande , og den Venlighed og Oprig ^ tighed , som hersker , vcere dog fjern fra en i Stovet neddragende Fortrulighed . Hendes Myndighed bor fslges af sieblikkelig Lydighed , af hvad Alder hendes Born end maa vcere ; hendes <b>Visdom</b> bor vinde deres Tillio , hendes Kjcerlighed deres Gjenkjcerlighed og Taknemlighed . Hun maa saaledes vcere deres Selstat » , Raadgiver og Trsst og ved sin egen Dprigtighed opmuntre deres . De maa behandles saaledes , at de ikke snste at have vigtigere Hemmeligheder for hende . Iscer burde hun i sit Vcesen <b>vise</b> dem al den sande Gudsfrygts Renhed , Ydmyghed , Fasthed Stjonhed og Inde , dens rensende , ydmygende og helliggjsrende Kraft i Medgangen samt dens himmelske Hjcelp og Husvalelse i Modgangen , for at hendes Born ved hendes Exempel kunne vindes for Gudsfrygt , og de saaledes maa komme til ikte blot paa Jorden og her i Livet , men i Himmelen og for alle Evigheder , at prise hende salig .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015090808025'target=_>, 1876, Sandheden i Kristus</a><h4>773</h4>for de <b>Vise</b> , dåden kjcere Stjerne atter lyste for dem : thi der de saa Htzernen , bleve de ganste meget qlade . Hvem kan tmvle derpaa , naar efter en dunkel Nat , saaledes som de fandt den : Jerusalem , saadan en klar Stjerne atter ovqaar og v : er den Kge Vei til <b>Maalet</b> ? Hvo glceder sig ikke , naar efter tunge Kampe med Tvivl eller Modsigelser , Anfeqtelser og Hmdrmger , Hjertet atter stjcmkes Vished og Klarhed i den sendte Sandhed ? De have i Jerusalem af Guds Ord , af Vlbelen , faaet den Overbevisning , at han maatte vcere fM , oq hvor han maatte ftdes , og nu se de igjen det skinnende Tegn paa Himmelen . Da levede deres Hjerter op igjen , som om de allerede havde fundet den lcengst Forventede , den ivrig SMe . Jo ncermere man kommer ham , jo mere glceder Hjertet sig . O maatte dog Alle ftge ham saa ivrigt ! Gud vil ikke lade det ferle paa Stjerner , men lade for Enhver et klart Lys opgaa af sit Ord , i Hjertet eller paa Himmelen , for at drage og ftre ham tll SMnen . Hvo , som viser dig Jesus , forer og leder dig til Jesus , er for dig en Stjerne , sendt dig af Gud , det vcrre nu et Menneske , en Bog , eller hvilketsomhelft andet Guds VcerktK . O du Herrens , Frelsens og Naadms Stjerne , opgaa for alle dem , der endnu sidde i Mrke oq Dydens Skygge , aabenbar dig for Alle , der Jesus , frelseren, og ikke kunne finde ham ! « w de gik ind i Huset og fandt Barnet med dets <b>Moder</b> Mana og faldt ned og tilbade det oq oplode sine Forraad og ofrede det Gaver , Guld oq Roqelse og Myrrha . Hvilken Finden , hvilket Syn ! Den AllerhMstes SM , fM af en fattig Jomfru , i Menneskeskikkelse ! Israels lcenge ftgte Konge ! Der ligger han paa en Moders SHK ! Me mere i Stalden , men i et Hus sinde de ham , ved hvem Alt er blevet til , og uden hvem Intet er blevet til as Alt , hvad der er i Himmelen og paa Jorden . Hvorledes maa de trofaste , ufortrødne Sugere vcere blevne tilmode ! hvor have deres ss ) ine glindset , hvorledes er deres Hjerte blevet overstrømmet med Guds Fred , da de for ftrste Gang efter saa lang , mKsommelig Spgen saa ham , som deres Sjcel useet elskede ! Hvo vilde ikke gjerne dele denne Glcede , denne Fred , denne HK Salighed med dem ? Lad os ogsaa saa ivng , saa tro og ufortrøden vedblive at Lad os ogsaa

<h4>4305</h4>vare bundne ; han kunde ikke mere gaa omkring og vel . Hans Hcrnder vare bundne ; han kunde ikke mere faa rM ved de Syge for at helbrede dem . Alene Tungen var saavidt fri , at han endnu kunde rM den . Nu , saa vilde han da gjM Godt med den , naar han ikke kunde faa gjM andet Godt . Med den vilde han dele Naade nd , med den trMe sin <b>Moder</b> , med den styrke RMeren , med den bede for sine Fiender . Saaledes vilde han , som altid gjorde Godt , vedblive at Godt til sin DFd ; og deraf blev han ikke troet . Man betcenke dog , hvor uendelig denne hans Mistnndhed var . De faarede ham , han vilde lcege dem . Derfor beder han : Fader , forlad dem ! De drcebte ham , han vilde gjFre dem levende : Forlad dem ! De hadede ham , han elskede dem . De paaftrte ham Lidelsen , og dog bad han for dem ; de regnede ham blandt Misdcedere , han vilde dog vcere deres VelgjFrer ; de overantvordede ham til men han talede Livets Ord for dem ; de gjorde ham det Vcerste , de kunde ofttcenke , han bad for dem om det Bedste , han kunde bede ; de vilde ikke gaa fra Stedet , forend de kunde se ham dFd , og han vil ikke fM han vidste dem forsonet med Gud ; de havde eudnu deres Lyst i at se deres stMste Synd fuldfort , men hans Lyst var at vide , at deres Synd var forladt . Se dog , hvorledes han selv Mr , hvad han har lcert Andre at gjFre . Han fordrer Intet af os uden det , vi se ham selv gjM . Det var hans egen <b>Lcerdom</b> : Velsigner dem , som Eder bande , og gjsrer dem Godt , som Gder hade , beder for dem , som overfalde og forfolge Gder ( Matlh . 5 , 14 ) . Nn , det er altid lettere at sige Andre , hvad de stulle gjM , end at gjFre det selv . Jesus vil her <b>vise</b> , at han selv gjorde , hvad han lcerte Andre at gjM . Nu blev han hadet , slaaet , forfulgt , forbandet , ja ynkelig drcebt . Og se ! han taaler det , og han beder , han velsigner , han Godt . Samuels Sagtmodighed var visselig stor , da Israel forWd hans Regjering , var kjed af hans HerredMme og ncesten scetter ham af fra hans Embede . Det gjorde ham jo Ondt ; han klagede det og for Gud ; men han dog deres Bedste og forpligtede sig dertil , sigende : Det vcere langt fra mig saa at synde mod Herren , at lade af at bede ydmygeligen for Eder ( 1 Sam . 12 , 23 ) . Hans Sagtmodighed var stor , dog ikke

<h4>4678</h4>Nei , Guds Raad augrer ham aldrig og allermindst nu , da det var saa godt som overstaaet . Men han spurgte for dermed at give tilkjende , hvad han havde lidt , og hvor græsselige disse tre Timer havde vceret , hvilket vi ellers aldrig havde kunnet tro , tcenke eller falde vaa , uden at vi saaledes havde HM det af lesn egen Mnnd . Han spørger derom , for at vi stnlde vide det og med Flid eftertcenke , hvad det har at sige , at han var forladt af Gnd . Han spsrger hvorfor for at <b>vise</b> os , at han aldrig selv har gjort Noget hertil , aldrig forbrudt sig i Noget , hvorved han kunde fortjene at forlades af Gud . Han har aldrig felv forladt Gud eller veget fra hans Vilje . Han spørger derom , for at vi skulle gaa iud i os felv og tcenke : Ak , Herre lefus ! Hvorfor er dn bleven forladt uden for min Skyld ? Fordi vi have forladt Gud , fordi vore Synder have gjort Skilsmisse mellem Gud og os ( Es . 59 , 2 ) . Vi have fortjent evig at forlades af Gnd ifplge den Dom : Spger dn Herren , skal du finde ham , men forlader dn ham , skal han bortkaste dig evindelig . Vi have Alle i Adam forladt Gnd , dengang han i Paradiset overtraadte Gnds Bud , og vi have siden selv saa tidt i egen Person ved mange Slags Synder forladt Herren og stnlle derfor evig have vceret forladt . Derfor maatte han , som er gaact i Borgen for os , betale det , han ikke havde rMt , han maatte udstaa den Forladthed af Gud , som vi havde fortjent , for at vi aldrig sinlde forlades . Saalcenge vi da ikke forlade dig , Herre lefus , men holde os til dig i Troen , saalcenge kan Gud ikke mere forlade os , end en <b>Moder</b> kan glemme sit diende Barn , ja om hun end kuude glemme det , saa vil han ikke glemme os ; thi han har forladt sin SM for vor Skyld og taget Fyldest af ham , og derved evig svoret og besluttet , at alle de , som tro vaa ham , aldrig stulle forlades , aldrig fortabes , aldrig fordMmes , aldrig blive forlorue , men have det evige Liv ( Joh . 3 , 16 ) . Se , det vilde han med dette Svxirgsmaal give dig og mig og os Alle at betcenke , og det Svar , som han ikke fik , ikke ventede af sm himmelske Fader , det venter han af os . O , for vor Skyld , o Jesus ! blev du forladt ; og derved ledes vi da til at fmde den Trsst og <b>Lwrdom</b> , som vi efter Scedvane til Slutning ville hente os af dette lesn Ord . Dette Ord er da visselig ligesaa trsfteligt for os , som

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014082608023'target=_>James, John Angell, 1872, unge Kvinde</a><h4>26</h4>Af Originalen have vi allerede uddelt mange Exemplarer til unge Venner og Bekjendte med Opfordring , at de ret opmcrksomt maatte gjennemloese og tcenke over denne Bog , og har lwnge ncrret dctOnfle , at den maatte blive oversat og ndgiven paa vort Modersmaal. Det er derfor med Gloede , vi nu se dette Onske opfyldt , Familielivets , og derved hele Samfundslivets , suude Udvikling og sande Velfcrrd har sine fsrste og dybeste Rsdder i Moderhjemmet . Det er fra Moderen , at Barnet faar sine forste , fleste og varigste Indtryk og Grundlaget til sin Udvikling og <b>Dannelse</b> for hele Livet , ja , for en stor Del for Evigheden er i Hjemmet , i Barndommens Morgen , at Barnets Karakter egentlig dannes , og intet kan til dette Oiemed vcere vigtigere , end at <b>Kvinden</b> , som har den store Bestemmelse at give de Slegter . som sodes , den fsrste stjoermende Pleie og at sprede den sande Kjcerlighcds Sollys og gyde den sande Fromheds Morgendug over Familielivets Hjerte , blade , - at hun dertil bliver forberedt ved Herrens <b>Aand</b> ; og i denne Henseende er en god , en udmerket Veiledning af en hsit oplyst og erfaren Lcrrer af megen Vcrgt .

<h4>28</h4>Fremdeles er det for Familielivet , saavelsom for Samfundslivet, übeskrivelig stjsnt og velgjsrende , naar <b>Kvinden</b> ved en rigtig Ledelse er dannet og forberedt til at opfylde sit vigtige Kald som <b>Hustru</b> for at kunne vcere Hjemmets storste Prydelse og den virksomme, milde Annd , fra hvilken det huslige Liv henter Trivsel og Lykke under den sande Kjoerligheds , den oplyste Forstands og den oprigtige Bons Magt , som Apostelen siger : „ Hjertets skjulte Menneske i en sagtmodig og stille Aands uforkrcenkelige Vcesen , hvilket er meget kosteligt for Gud . " 1 Pet . 3 , 1 - 5. Det er ogsaa ved en faaledes beskaffen <b>Dannelse</b> og Sjeelsretning , at den unge <b>Kvinde</b> , saavel som Mennesket i Almindelighed , ikke blot for sin egen Del bliver allerlykkeligst bande i Feedrehjemmet og under Udsvelseu af de forskjellige Kald i det almindelige Liv , men ogsaa faar Sind og Evne til at medvirke til andres Velfoerd og til at have en velgjsrende Indflydelse paa mange . I alle Henseender giver denne Bog fortrceffelig Undervisning og Vejledning . At en saadan kan tiltrænges, var let at <b>vise</b> ved Paapegelse af mange fsrgelige , fljoeve Retninger , der ere meget sedvanlige , isner i Henseende til den <b>kvindelige</b> Ungdoms Opdragelse , eller Mange ! paa Opdragelse , og den urigtige Behandling , den faar , forudeu mange Afveie , som ere at beklage .

<h4>60</h4>Til Hedenskabets og Muhamedanismens undertrykkende og grumme Behandling af Kvindekønnet fremstiller Kristendommen en skjM og elskelig Modsætning , idet den til Jødedommens delvise Bestemmelser angaacnde <b>Kvindens</b> Rettigheder tilftier en fuldkommen Erkendelse af hendes Fordringer . Det er vor hellige Religions Mrc og et Bevis paa , at den er udsprungen af den guddommelige Kjcerlighed , at den er en Fiende af al Undertrykkelse , at den giver enhver , hvad ham tilkommer . Den troedcr ingens Rettigheder under Fadder , den modstaar alt ondt ; men ingen staar i Mrre Forpligtelse til den end <b>Kvinden</b> for dens milde og retfærdige Styrelse . Ved Kristendommen har <b>Kvinden</b> erholdt sin moralske og sociale Indflydelse, ja ncesten hele sin Tilværelse som et Samfundets Medlem . <b>Kvindens</b> Sjcel , hvilken mange af Fortidens Filosofer , Lovgivere og <b>vise</b> ansaa for at vcere af ringere Vcerd og ude af Staud til at modtage <b>Dannelse</b> , er dog bleven udviklet under Kristendommens Beskyttelse . Kristi Evangelium , dette glade Befrielsens Budskab , er nedsteget i denne forsømte Grube , hvilken endog de <b>vise</b> have foragtet at oparbeide , og har derfra hentet op mangen ufkatterlig Juvel , straalcnde af Snillets Ild og af den kristelige Kjcerlighcds elskelige Farveglans . Kristendommen har gjcngivet <b>Kvinden</b> hendes tabte eller bortsvede Rettigheder og indsat de klareste Mdelstene i hendes Høderskrone. Hendes tidligere Forfald forklarer idctmindste for en Del Übestandigheden af tidligere Tiders <b>Dannelse</b> . Det er umuligt for Samfuudet vedvarende at holde sig oppe , naar dets <b>Kvinder</b> cre fornedrede til Slaver . Hvorsomhelst den <b>kvindelige</b> Slcegt ansees for underordnet , der bcerer Samfundet Oplosningens Spire i sit SkjFd og er berøvet Evnen til alt virkeligt Fremskridt . Det er umnligt , at Indretninger og Scedvaner , hvilke hindre eller kvcele Naturens Instinkter og krcenke Guds Love , kunne trones med endelig Fremgang . Samfundet maa forandre sit ydre Udseende , maa fremstille Rigdommens Glans , Smagens Forfinelse , Kunstens Forskønnelse eller Videnskab ens og Literaturens mere værdifulde Fremskridt ; men hvis <b>Kvindens</b> Sjcel forbliver

<h4>76</h4>det lunefulde godtbefindcndes forstyrrende Indflydelse . Hvor bsr hun ikke fryde sig over det Ord : „ Hvo , fom skiller sig fra sin <b>Hustru</b> uden for Hors Sag , gjM , at hun bedriver Hor , og hvo , som tager en fraskilt tilcegte , bedriver Hor . " IFdernl ' s stirre Frihed i denne Henseende blev saaledes ikke billiget af Kristendommen ; stMe Sikkerhed gaves <b>Kvindens</b> Mre og Lykke , og en dybere Grund lagdes for den huslige Lykfalighed . Om det ikke var for noget andet end dette , fortjente Kristendommen Menneskeslægtens Taknemmelighed. Men det er blot en Del af dens Mre , at den afskaffede Brugen af mange Hustruer ; endnu mere er det , at den beskytter denne ene Hustrus Rettigheder , Anseelse og Lykke . Den beskytter hendes Hjem for utilbørlig Indtrængen og sikrer hende selv en tryg og urokkelig Besiddelse deraf . Jeg vil ogsaa omtale den lige Andel i Religionens Velsignelser , som stjenkes den kristne <b>Kvinde</b> . Hvor tydelig og bestemt har ' ikke Apostelen tilsikret ogsaa <b>Kvinden</b> Del i ' alle Kristi Velgjerning er , naar han siger : „ Her er Ne eller Grceker ; her er ikke Trwl eller fri ; her er ikke Mand eller <b>Kvinde</b> ; thi I ere alle en i Kristo Jesu . " Dette er et Fribrev , som skjenker <b>Kvinden</b> alle Frelsens Velsignelser; dette er Beviset for <b>Kvindens</b> levnlighed med Manden for Gud ; dette er <b>Kvindens</b> Rettighed til sin sande Plads i Samfundet . Der findes ingen Velsignelse , nodvendig for at vinde det evige Liv , hvilken ei i famme <b>Maal</b> tildeles <b>Kvinden</b> fom Manden . Det er en almindelig Tro blandt Muhamedanerne , at <b>Kvinder</b> ikke faa Adgang til Paradiset ; ere istcdet scerlig skabte for denne Plads . Hvor fornedrende er ei en saadan Tanke ! Men den er i fuldkommen Overensstemmelse med Muhamedanismens <b>Aand</b> , der betragter <b>Kvinden</b> mere som en Slave for Mandens Lidenskaber end som hans Ledsagerinde gjennem Livet . Kristendommen stiller <b>Kvinden</b> ved Mandens Side , som <b>Hustru</b> , som Datter , som Tjenestepige ; ved Familiearnen, i Templet , i Menigheden . Mand og <b>Kvinde</b> mFdes

<h4>81</h4>og bestandig at opholde sig i hans Noerhed , var den mindste af de ' Udmoerkelser , som vare bestemte for dem af vor hellige Religion . De indtoge en fremragende Plads i de Arbeider og Sysler , som paakaldtes af Nødvendigheden i de apostoliske Tider for Udbredelsen af Kristi Rige paa Jorden . Hvor lcererig og rerende er det ikke at HM Pauli Opfordring : „ Antag dig disfe <b>Kvinder</b> , thi de have stridt med mig i Evangeliet . " Hvilken Moengde af Navne paa udmcerkede <b>Kvinder</b> og deres Arbeider sinde vi ikke i Romerbrevets 16 de Kapitel . „ Priscilla , min Medarbeider " ; „ Maria , som har arbeidet meget for os " ; „ Tryphena og Tryphosa , som have arbeidet i Herren " ; „ Phoebe , som er Menighedens <b>Tjenerinde</b> i Kenchrece " , der sendtes til Menigheden i Rom og betroedes en saa vigtig Sendelse , som at overbringe den det af Apostelens Skrifter , hvilket , om vi have Lov til at fammenligne den ene Del af Skriften med den anden , er den mest dyrebare af den guddommelige Aabenbaring ! I Forbindelse med alt dette er det udenfor al Tvivl , at <b>Kvinderne</b> i den ftrste Menigheds Tider ilte alene begavedes med Aandcns underbare Kraft at profetere , men ogsaa anvendtes som Diakonisser . Nyere Tiders kristne Kirker ere ikke vedblevne med at cere den <b>kvindelige</b> Karakter . Kristendommens Barndom stod i denne Henseende foran vor Tid ved at anvende <b>Kvinden</b> i Kirkens og Menighedens Tjeneste og i den <b>Visdom</b> , hvormed den forstod at benytte sig af deres rige og anvendelige Hjælpekilder . Det er blevet bemcerkct , at Samfundets Seeder for en Del ere forandrede siden den Tid , saa at de have gjort <b>Kvindens</b> Betjening af sit eget KjM mindre nødvendig nu , end det var den Gang , da det venlige og fortrolige Samfund mellem Kjsnnene var mere bundet , og <b>Kvinderne</b> holdtes i stirre Enfomhed og Affondring . Der ligger uden Tvivl nogen Sandhed heri , men maaste ikke saa megen , som nogle Mfle at tro . Vaade den almindelige og den hellige Historie fremstiller disse Tiders <b>Kvinder</b> som saadanne , der arbeidede i Samfundet og deltoge i Mandens Beskæftigelser . Jeg bemcerker nu , at Kristendommen ved sin <b>Aand</b> og sine Forskrifter ikke blot saaledes vil hoeve den kvindelige

<h4>89</h4>til yderlighed overdreven Vurdering af <b>Kvindens</b> Rettigheder og Forrettigheder end en bevist Indrvmmclse af dem , betragtede og opfattede af Fornuftens Lys og i Overensstemmelse med Guds Bud og Forskrifter ; det kan ogsaa vcerc et Sporgsmaal , om ikke mangen tapper Ridder , naar Indbildningens Glans var blegnet og Lidenskabens Ild udslukket , kunde knuse det Hjerte , han havde veeret saa ivrig for at mnde , og dette just fordi Kristendommens <b>Aand</b> var ham fremmed . Tet er Kristendommens Mre , at den , istedet for at virke paa Indbildningen , Sanserne og Lidenskaberne, meddeler en <b>Aand</b> , som tager Bolig i Sjoelen og behersker Samvittigheden , og iftedet for at forfsre Manden til taabelig Beundring af <b>Kvindens</b> Inde ved offentlige Forlystelsers pirrende Optrm , leerer og leder ham til fsrst og sremst at betragte hende og nyde hendes Omgang i Familielivets übemcerkede Fortrolighed og i Hjemmets hellige Kreds , Kristendommen tillader ikke dette taabelige Afguderi , som Riddervcesenct befordrede , og fom just paa Grund af fin yderlighed maatte esterfolges af Ligegyldighed eller Foragt . Kristendommen gjor af <b>Kvinden</b> hverken en Guddom eller en Slave den gjor hende til Mandens Ven og Ledfagerinde gjennem Livet og leorer ham at betragte hende i dette hcedrende og elskelige Lys . Kunne vi ikke i alt dette se et skjsnt Bevis for vor hellige Religions umaadelige Fortrin ? I enhver Henseende , om vi betragte dens Virkninger for denne eller den tilkommende Verden , dens Forhold til Samfundet etter til Gud , dens hoie Lcerdomme eller dens rene Moral , se vi her et Billede af uefterlignclig Skjpnhed , mcegtigt til at indtage ethvert Hjerte , fom ikke er blevet forstenet ved falsk Filosofi, aabenbar Vantro eller grov llscedelighed . Men i ingen Henseende er det skjonncre end med Hensyn til <b>Kvinden</b> . Med hvilken Ligelighed afveicr den tkke alt mellem begge Kjon ! Med hvilken Mrldhed holder den tkke sit Skjold over det svagere Kar ! Med hvilken <b>Visdom</b> opretholder den ikke og forsvarer deres Stilling og Rettigheder, hvis Indflydelse er faa vigiig for Samfundet , og begrcondser alligevel deres Fordringer faaledcs , at de ikke

<h4>106</h4>Naar man betragter <b>Kvindens</b> vigtige Plads i den store Familie , som kaldes Menneskeheden , vilde det vcere besynderligt , om der i et Skrift , indblcest af Guds <b>Aand</b> til Menneskehedens Veiledning og Lyksaliggørelse , om der ikke , siger jeg , skulde vcere anvist <b>Kvinden</b> en Plads , som svarede til den Indflydelse , hun er skabt til at udMe . Bibelen giver os en Beretning om Samfundets Oprindelse og Sammenfletning og har blandt andre og endnu hMre <b>Formaal</b> ogsaa det , at lede dets Udvikling og befordre dets Lykke . Dette kunde den ikke hvis den udelukkede <b>Kvinden</b> eller stillede hende blot i Baggrunden uden Hcnfyn til hendes vcrlende Forhold , hendes Kald og hendes Pligter . Det Emne , som vil blive Gjenstand for Betragtning i dette Kapitel , berørtes allerede i det forcgaaeude ; det skal her blive udførligere afhandlet . Manden er naturligvis sMst og fremst Gjenstand for den llllbenbarede Sandhed . Der ligefom i Samfundet indtager han den ftrste Plads . Der siges der mere om , til og ved ham , end hvad der Kenftres til <b>Kvinden</b> . Han er den fornemste , om end ikke den eneste handlende Person i det store Drama , som Forsynet afslører for os paa den hellige Bogs Blade . Hans Ledsagerinde fremstilles ogsaa , hverken stillet i Skyggen eller omtalt blot tilsceldigvis . Hendes Navn og Historie , hendes Dyder og Laster , hendes Sorger og Sysler indtage et anseeligt Rum i den hellige Bog . Hun kan ikke klage over , at hun er udeglemt eller lagt tilside i en Vraa ; der er sagt tilstrækkeligt om hende for at gjore hende tilfreds . Hun bs ' r derfor vcere taknemlig, og uden guddommelig Naade vilde hun kunne fristes til at blive stolt derover . Hun bM ikke kunne tabe Agtelsen for sig selv , ikke heller andres Agtelse , just fordi hun er saaledes omtalt og behandlet i Skriften . I denne Henseende udgjM Bibelen en skjM og herlig Modscetning til Koranen . Wrst vil jeg betragte den Beretning , Bibelen meddeler om <b>Kvindens</b> Skabelse og Fald , og jeg vil i forbigaaende bemcerke , at det er Aabenbaringen og den alene , som vi have at takke for vor <b>Kundskab</b> om Menneskeslægtens Opkomst.

<h4>173</h4>Kristendommens , Merlandenes eller Vesterlandenes , Fortidens eller Nutidens , og vi skulle finde , at intet saa voesentlig adskiller dem som <b>Kvindens</b> Tilstand . Enhver <b>Kvinde</b> , hun vcere nu gift eller ugift , rig eller fattig , har dog en Kreds , indenfor hvilken hun udover en <b>vis</b> Magt , enten til Skade eller Gavn . Enhver <b>Kvinde</b> vil nødvendigvis ved sin Daarstab eller sin Klogskab , sin Dyd eller sine Laster forege sit Folks Vcerdighed eller paaskynde dets Forfald . Saalcengc <b>kvindelig</b> Dyd blomstrer , beskyttet af det ene og agtet af det andet KjM , kan et Folk ikke synke altfor dybt . Til en <b>vis</b> Grad er det <b>Kvinden</b> , som opretholder sit Folks Lykke . Hendes Dyd staar , hvis den er fast og ufordærvet, som en beskyttende Vagt over Folket . Lov , Retfærdighed og Frihed , Kunst og Videnskab , alt bidrager naturligvis til et Folks Velgaaende , men hvis <b>Kvindens</b> almindelige Sædelighed staar saa er alt andet sorgjceves , medens derimod , hvis <b>kvindelig</b> Dyd og <b>Dannelse</b> er almindelig, Civilisationens Stro ' m stiger til sin Drste og bliver frugtbringende for den mest udstrakte Overflade . Et Samfund , hvori <b>Kvinden</b> opfylder sit Kald , falder ikke lettelig ; thi den Magt , hendes cedlc Hjerte udMer paa andre Hjerter , vil lMe det af Elendigheden og gjengive det Glcede og Lykke . Se der , faavcl udenfor fom indenfor det ægteskabelige Liv hur <b>Kvinden</b> en Mission , og det ei en übetydelig . Hendes Virkekreds er Samsuudet , hendes <b>Maal</b> er dets Velbefindende , hendes BelMning dets Taknemlighed og Erkendtlighed .

<h4>183</h4>Land . Og angaaendc de mangfoldige Stiftelser for Religionens og <b>Dannelsens</b> Udbredelse og Nodens Afhjcelpelse , paa hvilke vor Tidsalder er saa rig , saa giver jeg hende frit Rum til at virke der saa meget , hun kan , naar det sker uden Men for den egne Husbestyrelses Pligter . Hvilken <b>vis</b> <b>Kvinde</b> kunde Mste mere ? Hvilken Forsvarer af hendes Rettigheder skulde ville gjFre Fordring paa mere ? <b>Kvinden</b> er altid paa sin Plads , hvor Velgørenheden hersker, nndtagen naar hendes Tid og Opmærksomhed udfordres i Hjemmet for dem , som fremfor alt andet have Fordring paa hendes Opmærksomhed . Men jeg stal nedenfor videre komme tilbage til denne Gjenstand . Men hvad stal man sige om de <b>Kvinder</b> , som gjpre Fordring Paa de Vestjceftigclser , Pligter og Rettigheder , som soedvanlig betragtes som tilhorende udelukkende Manden? Der fiudes nogle , som ville snske at overskride den Grcendselinje , hvilken ncestcn alle Folkeslag have optrukket mellem Mandens og <b>Kvindens</b> Pligter og Beskæftigelser . Kristendommen har henvist Kvmden til en Plads , hvor hun passer , en Virkekreds , hvor hun kan baade lyse og oplyse . Forflyttet fra denne Kreds kan hun blot sprede et svagt , blegt Skin omkring sig . <b>Kvinden</b> er lig en Plante , som i sit eget Blomsterhus henrykker med den lifligste Duft og de skjønneste Farver , men ' som flytter udenfor dets Beskyttelse paa Verdens Tumleplads snart visner og blegner . Hverken Fornuften eller Kristendommen indbyder <b>Kvinden</b> til Lcererstolen eller Dommersædet ; ingen af dem kan siges bestemt at udelukke hende dersra , men saavel deres almindelige Anskuelse som den almindelige Mening forvise hende derfra . De hceve sin advarende Stemme , at hun ikke maa bortkaste sit KjMs Blufcrrdighed og lytte til Stemmer , som fordre nye Rettigheder for hende ; hun vogte sig for at blive deres Rov , " som fremstille sig som hendes Kjcemper blot for at vinde Navnkundighed eller maaste ikkun Vr , sd. Bibelen giver <b>Kvinden</b> sin Plads i Hjemmet , en Plads af Bcerdighcd og Majestoet , i sin Mgtefcelles og sine Bsrns Hjerter ; der / hvor Kjcerlighed , Amhcd , Omhu , Wlelse og Eftertanke herske , der ligger hendes Herredomme .

<h4>215</h4>Og hvis det ikke cmsees for utilladeligt for nogle Aieblikt ' e her at gjMe en liden Afstikker og ogsaa tiltale Fcedrene , saa vilde jeg sige dem , at ogsaa de bM dybt og alvorlig betcenke Vigtigheden af de huslige Forhold og deres Indflydelse paa Samfundet . Hvor meget der end bliver fagt og strevet i denne Tid om Forbedringer i Samfundet , man man dog aldrig forglemme , at al virkelig Forbedring maa begynde i Hjemmet og i vore Familiers Hjerter . Alle Forsag Paa at helbrede det onde eller erstatte Manglerne ville , hvis de ikke begynde der , blot lede til overfladiske og utilfredsstillende Resultater . Det er kun i den rigtige Opfattelse af Forceldrenes Pligter og i Mgtefcellcrncs rette Udøvelse af sine Pligter indbyrdes og mod Vprnene , at man bM ftge det Lcegcmiddel , der kan helbrede Folkets Snar faavelsom opholde dets Sundhed . Anstalter kunue reise sig som en Bistand eller Erstatning for Forceldrenes Pleie eller for at udvide eller foraudre <b>Kvindens</b> Virkekreds , og lad FKgen blive et tilkunstlet Samfundsliv , Pralende med en tom Forgyldning af menneskelig <b>Visdom</b> , men man skal dog til Slutningen finde , hvor umuligt det er at tilintetgøre de store Formaal, hvilke Gud , den store Ophavsmand til det menneskelige Samfund , har tilsigtet ; Forcrldrcne maa trods alle Hindringer vcere Værnenes Lcerere og Hjemmet den hvor Karakteren uddauncs . Den unge <b>Kvinde</b> bsr ikke lades i Uvidenhed om ut Kald . Hendes fremtidige Stilling bs ' r af en <b>vis</b> <b>Moder</b> eller <b>Lcererinde</b> foreholdes hende , forat hun maa ihukomme Nødvendigheden af at berede sig for sin tilkommende Lod . Huu maa paamindes om , hvad der er mere end al <b>Lcerdom</b> , at hendes Karakter maa uddannes , et <b>Maal</b> , som ikke kan ovnaaes nden hendes egne Anstrengelser . Hun maa tidlig lcere sig selv , at hun har et Kald paa Jorden , vcerd al mulig Opmærksomhed og Forberedelse , ikke for at hendes Forfængelighed derved stal opblceses ,

<h4>222</h4>Uddannelsen af Talenter maa vcere underordnet vigtigere Anliggender , og ligeledes bM Legemets Udstyrelse agtes ringe og übetydelig af eder i Sammenligning med Sjoelens og Hjertets Prydelse ! Som en Forberedelse for en lettere Opfyldelse af eders fremtidige Kald og fom en Betingelse for eders Wgtefcelles Tilfredshed maa I ogfaa vcenne eder til Opmærksomhed, Orden og Disse kunne vel kaldes mindre Dyder , men ere dog af stor Vigtighed for en Husbestyrelse , og Savnet af dem bringer Hjemmet i en Tilstand af Forvirring og virker nedslaaende paa Sindsstemningen. Kommer Vanens Magt ihu , og at gode Vaner maa erhverves i Ungdommen . Strceber efter en <b>vis</b> Fuldkommenhed i det hele . Gjerer Smaating godt . Hvis en Ting er vcerd at gjMe , er den ogsaa vcerd at gjM godt . Undviger Uagtsomhed i det mindre saavel som i det Drre . Unge <b>Kvinder</b> , hele eders Fremtids Liv vil bekrcefte Sandheden af disse Rand , enten ved det Velbefindende , som I ved eders Lydighed mod dem flabe omkring eder , eller ved de Sorger og Lidelser , som Forglemmelsen af dem vil paadrage eder felv og andre ! Det er alene paa denne Vei , at I paa en velsignelsesrig Mande kunne opfylde, hvad vi i dette Kapitel have sM at fremstille , <b>Kvindens</b> Kald i Samfundet .

<h4>266</h4>eders Ansvar . I ere ikke ved Lasten naaede til en haarb ikke heller ved gjentagne Synder nedsunkne i nlykkelige Vaners Baand , ikke heller har Vantroens forhærdende Magt gjort eder sor Religionens Indflydelse. Eders moralske Melse bliver ikke ved en lang Rcekke af Synder saa slavet , at den ikke holder Veien til eders Hjerter aaben for Advarselens RM . Og derefter , betcenker endnu en Ting , eders ledige Tid . Jeg taler nu om <b>Piger</b> , som leve i Forceldrchufets Beskyttelse og ikke behøve at fortjeuc sit Ophold ved sit Arbeide . Eders Tid , med Undtagelse af hvad en forstandig <b>Moder</b> glr Fordring paa til Bistand i Husholdningen , er fcedvcmlig ganske eders egen . Eders Brødre , de maa nu vcere hjemme eller borte , maa nødvendigvis vcere mere sysselsatte . Deres Tid er ide foerreste Tilfcelde deres egen , noget , fom man altfor ofte fremhcever og endog anftrer fom Undskyldning for ForsMmclighed i , hvad som nngaar Religionens Fordringer og Sjcelens Frelse . I have ingen saadan Undskyldning . Eders Tid er saa meget eders egen , at I kunne gaa og komme , lcese eller arbeide efter eget Behag . I have endog faa megen Frihed , at Tiden undertiden fynes eder altfor lang , og jeg frygter for , at nogle af eder maaste forede nogle Timer hver Dag paa unyttigt Arbeide uden <b>Formaal</b> . I have mindst af alle Grund til at klage over Mangel paa Tid til at tcenke paa Evigheden eller Maugel paa Leilighed til at ftge eders Salighed . Er det muligt , at I kunne vlrrc blinde for eders ncervcerende Fortrin , at vcere fri for ncesten al Slags Bekymring ? I ville maaste engang sande Apostelens Ord : „ Der er Forstjel imellem Hustruen og Jomfruen . Den ugifte har Omhyggelighed for de Ting , som lMe Herren til , at hun kan vcere hellig baade paa Legeme og i <b>Aand</b> , men den gifte har Omhyggelighed for det , som hKcr Verden til , hvorledes hun kan behage Manden!" Hvor megen <b>Lcerdom</b> , Advarsel og Raad ligger ikke i disse faa Ord ! Apostelen vil ikke sige , at alle ugifte <b>Kvinder</b> handle faa , og vi vide desvcerre , at altfor mange af dem ikte spFrge efter at behage Herren , men hans

<h4>287</h4>til Fornsiclfer og Nydelser Wre eder Paa i Bedrøvelsen , i Fattigdommen , i Dsden ? Hos nogle indgiver Bevidstheden om den personlige SkjMhed en Egenkærlighed , som dem blinde for alt andet , og en bestandig TMst efter at blive lagt Merle til og beundrede . Ingen virkelig skjM <b>Kvinde</b> kan vcere nden <b>Bevidsthed</b> om sin natnrligc Inde , men kun altfor sjeldcn bor en skjM Sjcel under et skjMt ydre . Alligevel , hvor modbydelig er ikke en blot og bar ydre SkjMhed , naar den lun udgjFr Hyllet for en indre Hceslighed eller Sjcelens Vanskabthed . Hvilken SkjMhed kan vel sammenlignes med Sjcelens , og hvad er vel Sjallens hFieste SkjMhcd , hvis ikke Helligheden ? Det er Englenes SkjMhed ja , Guds egen . Hvor uforstandigt er det ikke at hovmode sig af den SkjMhed , som en ondanct Sygdom kan forvandle til hceslig Vanskabthed , og som , hvis ogsaa Sygdom ikke rMer den , dog fortoeres af Alderen og hensmuldrer i Graven ! Mangen <b>Kvinde</b> har allerede i . denne Verden havt Grund til at beklage sin henrivende Inde og til for evig at begrcedc den i en anden Verden . SkjMheden har for tusinder vceret en Gift baade for Legeme og Sjcel , for deres Fred og Salighed . Forfængeligheden rMer sig ogsaa i Omsorg for ydre Prydelser , endog der , hvor ingen Fordring findes paa ydre SkjMhed , og den forsoger ikke sjeldcn just derved at erstatte denne Mangel . Hvor mange tusinder bryde sig ikke langt mere om Pynt og Juveler eud om Religionen , den kostelige Perle ; mere om Smykker end om Retfærdighedens Klcedcbon og Salighedens Kjortel . Denne Kjcerlighed til Klcedcr er llkc alene en Svaghed , en undstyldelig Feil , men ganste vist en Synd , og har i utallige Tilfcelde ledet til Laster og Brode . Er det ikke ynkeligt at se et fornuftigt og udpdcligt Vceseu faa optaget og sysselsat med at Pryde et forgjcengettgt Legeme med forgjengelige Ting , uden et j2ieblik at tomte paa Ejcelen og hvad dens Prydelser burde vcere ; blot med dette til sit <b>Maal</b> , at syues indtagende for Menuesker , uden at fMge for , hvorledes den skal <b>vise</b> sig for Gud ?

<h4>294</h4>ind i Verden . Tcenker ikke altfor HM om eder selv , thi I ere Syndere , I saavelsom alle . I behøve , I ogsaa , og I kunne ogsaa vinde den Frelse , som er beredt i Jesus Kristus . Tcenker ikke heller altfor ringe om eder ; thi I ere udødelige Vcesener og kunne arve et evigt Liv . Hcever eder til eders sande Vcerdighed ved at hceve eder til en sand Gudsfrygt , Tro og et Liv i Sandheden . Forspilder ikke eders Liv i smaaligt , om ogsaa uskyldigt Tidsfordriv , uvcerdigt eders Evner , eders Bestemmelse og eders Pligt . Betragter med klart Blik og fuld <b>Bevidsthed</b> eders Kald for denne Verden og eders Bestemmelse for den tilkommende. Ser , hvorledes et Kald her aabner sig for eder , hvilket I maa opfylde med en kristens Hellighed , Mre og Velsignelse saavelsom med alle en <b>Kvindes</b> Dyder og Fortrin . Drager eders Hjerre fra Forfængeligheden , skanker det til Gudsfrygten . Skjcenker Gud eders Livs Morgen , og da stal dets Dag , den blive lang eller kort , klar eller overskyet , ensom eller tilbragt i Forening med andre , lys eller mFrk , rolig eller stormende , om den end ender ved en hastig HjemsFgelse , saadan som eders KjM vel kjender , eller om den gaar ned i en lang og mM Aften , den stal dog for eders falige <b>Aand</b> , befriet fra Stevets Byrde , afluses af den skyfri og evige Morgen , fom ikke efterfølges af nogen Nat . Da stulle I finde , at <b>Kvindens</b> faavelsom Mandens <b>Maal</b> var at forherlige Gud og nyde af hans Vcesen for evig .

<h4>303</h4>et vcrrdilM Ugrces istedetfor en Blomst , en Graasten istedetfor en Juvel , et Stykke tM Trce , flydende ud for StrFmmen istedetfor et levende Frugttrce , vorende ved dens Vred ! Ja , uslere end alt dette ; thi Ugrcesset , Stenen , Veden kunne dog paa en eller anden Maade anvendes til nyttige <b>Formaal</b> , men den , som selv ikke gjM noget godt , til hvad godt duer vel han uden til at vcere en Advarsel for andre ? Maatte da eders unge Hjerter banke af ønsket om at gj , sre godt , at vcere nyttige ! Strceber efter den ILre at gjMe godt . Vogter eder for at nedsynke eller svinde ind i en foragtelig Selviskhed : Udslukker ikke eders Atraa efter eu velgørende Virksomhed , og betragter det blot som et halvt Liv kun at leve for eder selv . I denne Henseende vilde jeg endog opvcekkc eder til Nidkjcerhed . Apostelen siger : „ Det er godt altid at vcere nidkjcer r det gode . " Nidkjcerhed betyder , som I vide , en inderlig Atraa , som virker en tilsvarende Kraft til at handle i en <b>vis</b> Hensigt og for et eftertragtet-Formaal , og naar denne Nidkjcerhed er rettet mod et rigtigt <b>Maal</b> , betegner den en cedel og oplMet Sindsstemning . Dette Sindelag fordrer imidlertid stor Forsigtighed i sin Virksomhed , iscer hos de uuge og fremfor alt hos <b>Kvinden</b> . Det ligner Ilden , som visselig kan benyttes til mange nyttige Formaal, naar den anvendes ret og passes , men hvis den forlades uden tilstrcel ' kelig Bevogtning og Tilsyn , kun foraarsager Ildsvaade .

<h4>311</h4>anvender undertiden de ringeste Midler til store <b>Formaal</b> , som jeg allerede har bemertet ved Beretningen om den lille Tjenestepige i Na ' amans Hus . Hedenskabet giver os her en <b>Loerdom</b> ; thi hvad sagde Herren til Profeten angaaende de hedenske Stikke , som Ilderne efterfulgte ? „ Ser du ikke , hvad de gjMe i luda Stcrder og paa Jerusalems Gader ? Børnene opsanke Trce , og Fcedrene optcende en Ild , og <b>Kvinderne</b> celte Deig at lave Himlens Dronning Kager / og udjZse Drikofre for andre Onder . " Hvilken Kappestrid ; alle Sind ere her optagne , alle Hcender ere i Virksomhed , <b>Kvinders</b> og BMns . Maatte vi drage <b>Lcerdom</b> af dette slette Erempel , og maatte vi <b>vise</b> samme Nidkjcerhed for den sande Gud , som de frafaldne IKer for de falske ! Kristendommen kan ligesaavel som Hedenskabet afgive en Mark for <b>Kvinder</b> og BMn til at bearbeide ; og hvor hellige ere ilke de Forpligtelfer , fom den paalcegger sine Bekjendere om at udbrede den . Efterdi ingen Tjeneste kan vel udftres af dem , som ikke ere skikkede dertil , vil jeg her fremstille nogle af de fornemste Vetingelfer for et heldigt Udfald af <b>Kvindens</b> Virksomhed for Religionens hellige Sag . Den religiøse eller aandelige Nidkjcerhed bFr altid udflyde af personlig Gudsfrygt / Foruden den kan ingen virkelig , varig Fremgang vindes . Noget kan vistnok udrettes; thi Gud kan jo sig til Nytte anvende deres Arbeide , som ikke have tilsigtet hans Mre . Men det er kun det Sind , der i Sandhed frygter Gud , fom kan fatte <b>Maalet</b> for en gudelig Nidkjcerhed , som kan ledes af rette Bevæggrunde, og som har mest Sandsynlighed for at vedblive i den gode Gjerning og at vinde Guds Velsignelse for sit Arbeide . Eders eget Hjerte maa vcere ret for Gud , ellers vil det kun lidt lykkes eder i eders Anstrengelse at formåa andre til at blive det . Ercmplet maa Mge med og stadfoeste Formaningen , ellers vil den sidste have liden Virkning . Meget af den ncervcerende Tids Virksomhed lider af Mangel vaa Idmyghed og aandeligt Alvor . Det er en sviende , pralende , ei af BMncn helliget Virksomhed. Der hersker hos altfor mange et Letsind ved dens

<h4>323</h4>Dette giver dobbelt Sjelsstyrke og Forvisning under Vandringen. Det os sterke ikke blot iog for os selv , men ogsall og i Særdeleshed i Herren og i hans Styrkes Kraft . Hvilket Mod skjenker ikke paa Kjcerlighedens Vei en Betragtning saadan som denne : „ Jeg ved , at jeg med rette Midler arbeider for et godt <b>Formaal</b> ; jeg vil gaa fremad i Herrens Kraft ! " Unge <b>Kvinder</b> , ogsaa i eders übemerkede Kreds , ved eders svage Vestrcebclser for at det gode skal en troende BMnens <b>Aand</b> , som i Sandhed cr den eneste , i hvilken alt burde foretages , bringe Englenes Gud til eders Hjelp og Herrernes Herre til eders Vistand . Vedbliver i den <b>Bevidsthed</b> , at I ret , og i den Tro , at den almcegtige gaar ved eders Side . Det betegner ikke Stolthed og Selvbedrag , men knn den Følelse as sin Evne , som enhver bM ncere . at sige : „ Jeg ved , og jeg Mer , at jeg er noget og kun udrette noget ; jeg behpver ikke at vere et Nul , et intet , thi Gud selv har ikke bestemt mig til intet . Jeg har <b>Sjel</b> og Hjerte , Vilje og Tunge , og med disse Gaver kan jeg noget for Gud og mine Medmennesker . Andre af min Alder og mit Kjs ' n , svage og ringe ligesom jeg , have dog udrettet noget , det kan ogsaa jeg og med Guds Hjelp og Velsignelse vil jeg det ogsaa . " Ja , du har Ret , dette er ganske sandt . Dette er Selvkundskab og en ret Agtelse for dig felv . Rer saadcmne Tanker , handle efter dem , og du stal udrette uoget . I en saadan Sindstilstand maa du skride til det vigtige Verk .

<h4>346</h4>saaledes optages af dette , at de blive uduelige for de Pligter, som vel i visfe Henseender ikke have faa meget tiltrækkende, men dog for dem ere af Drre Vigtighed . Jeg vil derfor advare unge <b>Fruentimmer</b> for i Ungdommen saaledes at vcenne sig til det offentlige , at det bliver dem til Men og Skade , naar de kaldes ttt det huslige Livs Pligter . Thi intet vilde bringe mere Vancere over Religionens hellige Sag , end om de lode dette falde sig til Last . Kjcerlighed til Virksomhed er et Gode ; Lidenskab derfor er derimod et Onde . Der gives en forstandig , cedruelig og bibelsk Nidkjcerhed ; der kan ogsaa gives en Nidkjcerhed , som blot er Kjcerlighed til Adspredelser under Religionens Skin . Naar en ung <b>Kvinde</b> elsker Hjemmet og dets Pligter , men er villig til ved passende Leilighed og for et vcerdigt <b>Formaal</b> at deltage i den ydre religiose Virksomhed og almindelige Velgørenhed , da har hun det rette Sind . Men hvis Hjemmet med dets Pligter forekommer hende kjedeligt , og hun stedse og altid lcenges efter ydre Virksomhed , da er allerede hendes Smag fordcervet , hendes Karakter stadet , og hendes fremtidige Udsigter til en <b>vis</b> Grad fordunklede . Hvis huu har forsaget den Tanke eller det > j2nste nogensinde at blive en gift <b>Kvinde</b> , og istedet , uden at tcenke paa at gaa i Kloster , dog har besluttet at blive en Barmhjertighedsftster , da kan det visselig vcere i sin Orden ganske at hengive sig til velgørende Virksomhed ; men hvis dette ikke er Tilfceldet , maa hun tage sig ivare for de Vaner , fom maaste engang ville gjFre hende uskikket og uduelig til at opfylde ægtestandens Pligter . Unge <b>Kvinder</b> saalcenge de forblive i Forceldrehuset, lade lsig lede af sine Forceldres , iscer sine Mudres Dnsker . De ' ere ikke , skulle heller ikke Mfie at vcere uafhcengige af Forceldreues Myndighed . En god og forstandig Datter vil altid med MrbMghed og Hengivenhed fe op til en god og <b>vis</b> <b>Moder</b> og vil derfor heller ikke begynde nogen religiøs Virksomhed nden hendes Raad og Bifald . Det kan hcende , at Barnets ønsker stundom stride mod Forceldrenes Synsmaade cmgaaende dette Punkt , det fordrer

<h4>374</h4>uvenligt , og deres Vaner uforenelige med huslig Fred , drage de dog snart bort , og Ufreden forsvinder med dem . I forblive derimod , hvis I ikke ere tvungne til at ftge Tjeneste for eders Ophold , hjemme hos eders Forceldre , indtil I skulle indtrcede i eders eget Hjem . Hvis I ikke fa « noget eget Hjem , forblive I hos Forceldrene for bestandig. I dette Tilfcelde blive I Fcedrehjcmmet tilhørende , og da er eders Forpligtelse til at forFge dets Hygge endnu stirre . Hvor megen Fred eller Ufred kau ikke en Datter , alt efter sit Karakter og sit Sindelag , afstedkomme i en Tid af ti til tyve Aar , regnet fra hendes Skoletid ! Derfor er det altid en Kilde , ikke til üblandet Fryd , men ogfaa til Uro og Frygt for en omhyggelig <b>Moder</b> , paa hvilken Maade hendes af Skolen udgangne Datter kommer til at arte sig , uaar hun vender tilbage til Hjemmet og efter afsluttede Studier ombytter sin Lcererindes og sine Kammeraters Selskab med Familiekredsen . Her er faalcdes det fyrste og vigtigste , som en nng <b>Pige</b> bM opstille som sit <b>Maal</b> ved sin Tilbagekomst til Hjemmet , nemlig saa meget som muligt at bidrage til den huslige Kredses Gloede og Hygge . Du kan der ikke vcere et Nul , du maa ikke tro , at du kan vcere der , uden ganske at udMe nogen Indflydelse. Det lille Samsuuds » vrige Medlemmer maa alle ftle Indvirkningen af din Opftrscl . Du er altid iblandt dem ; dine Handlinger , dine Ord , ja endog dine Blikke indvirke paa dem . Se der Begyndelsespunktet paa de huslige Pligters Bane . Fat da en fast , moden og bestemt Beslutning om med Guds Hjelp at gjpre dit Hjem saa lykkeligt, du kau . for andre , og derved ogsaa hyggeligt for dig selv . Indprceg denne Beslutning i dit Hjerte , din Hukommelse , din Samvittighed . Er den ikke <b>vis</b> , dydig og ret ? Billiges den ikke af Fornuften , Samvittigheden , Hjertet ? Gid den blive for dig en vigtig og alvorlig Gjenstand ; stjcenk den ikte blot en Tanke , fom flyver gjennem Sjcelen udeu at lade Spor efter sig , men lceg den dybt og alvorlig paa Hjertet . Desuden have jo dine Forceldre Ret til at vente sig dette . Er det ikke billigt , at

<h4>441</h4>hun man tage ud som <b>Lcererinde</b> , Vntikjomfru eller i Tjenerforhold . Madmsdres kolde , stolte og i nogle Tilfalde ogsaa grusomme Behandling , saa modsat Forældrenes og Overbærenhed , de og ligegyldige Kammerater, som udgjFr hendes Omgivelse , saa ulige hendes Søskendes kjcerlige og deltagende Omgjcengelse , den Tankeløshed og Ligegyldighed , som de fremmede , blandt hvilke hun opholder sig , <b>vise</b> for hendes Hygge og Tilfredshed , saa HM forskjellig fra den venlige Opmerksomhed , fom var scedvanlig i Hjemmets fredelige Kreds , alt dette er bittert , meget bittert . En haard og ukjcerlig eller en flygtig og letsindig Natur kan vcere ftleslos for alt faadant , men ak , det stakkels bIM , ftlfomme Boesen , hvis Hjerte er skabt for Hjemmets Gloede : hendes Hjerte er stundom fcerdigt at briste . Hvilke Tanker og Minder , lig Lyn af Lykke , oplyse ei hendes mMe og sørgelige Sti for strar igjen at forsvinde og gjM Veien endnu duuklere , MMet endnu mere trykkende ! Hvilke Breve , fugtede af Tnarer , skriver hun ei til sit eget Hjem , til sine Mme Forceldre ! Hvad har hun for Hygge ; hvad er hendes TrM ? Kun Religionens milde og lcegende Balsom , Bevidstheden om , at hun vandrer vaa Pligtens Vei og Samvittighedens Vidnesbyrd , at hun gjM sin Skyldighed med den største Troskab . Religionen kan og skal triste hende . Den , som har en sand Tro vaa Jesus Kristus , hvis Hjerte ledes af hans Helligaand , fom vandrer med Gud ligefom et Barn med sin Fader , ligesom en Ven med sin fortroligste , sin bedste Ven , hvis Kjcerlighed hcever sig over Jorden og dens sorgjcengelige Goder , og som betragter Livet blot fom en Forberedelse for Evigheden , hun vil ogfaa finde en Kilde til TrM , et kraftigt Middel til at bcere PrFvels n , et rigt <b>Maal</b> af Lykke , som vil overveie al MM , al Uro , al Sorg . Med sand Vcerdighed vil hun bM sig under Guds Vilje og betragte sin Stilling som bestemt af ham . Hun vil i Underkastelsen finde sin Glcede og Tilfredsstillelse. Religionen vil ikke blot stjcenke hende Taalmodighed, men ogsaa Lykke . Den vil beherske hendes Gemyt og skjcenke hendes Karakter en mild Inde , som vil

<h4>456</h4>Ved saadanne Anledninger vil du visselig kaste mange taarefulde Blikke tilbage paa det Hjem , fta hvilket du er bleven fordrevet af Ulykken eller af Forsynets Tilskikkelse ; gaa da til din himmelske Fader , og lad hans Haand aftMre dine Taarer , og lad hans Fred berolige dit oprørte Hjerte . Du har videre at bestjceftige dig med de BFrn , hvis Opdragelse er dig betroet . Betcenk , at dette Kald medfarer for dig et stort Anfvar for dem selv , for deres Forældre og for Gud . Afsky den Uredelighed , Grnsomhed og Lavhed , som er tilfredsstillet med sit Arbeide , hvorledes det end udfares , blot den betingede opnaaes . Disse <b>Pigers</b> Karakter og Fremtids Vel for tvende Verdener beror for en stor Del paa dig og paa den Maade , hvorpaa du opfylder dine Pligter . De ere blevne overladte til din Varetcegt og se op til dig fom sin <b>Lcererinde</b> og sit Exempel . Hvis du vil gjFre dit sidste Regnskab for Gud med Gloede og ikke med Sorg , gM da dit bedste ; anvend alle dine Kroefter og Evner paa at uddanne deres Karakter , udruste deres Forstand med <b>Kundskaber</b> og paa enhver Maade berede dem for den Plads , fom de ere bestemte til at indtage i Livet . Bered dig felv til dette Arbeide ved forsat Loesning og Selvstudium , veer ikke tilfredsstillet med den Dygtighed , du allerede besidder . Et fornuftigt og udpdeligt Boesens Opdragelse for denne og den tilkommende Verden er et vigtigt Verk . Beflit dig felv Paa <b>aandelig</b> Udvikling for at kunne befordre andres . SFg at vinde deres Tiltro ved din Duelighed , deres Hengivenhed ved din Benlighed , deres Agtelse ved din Vcerdighed og deres Loervillighed ved din egen Flid . Lad dit <b>Maal</b> vcere at meddele ikke blot <b>Kundskaber</b> , men ogsaa <b>Visdom</b> , ikke blot at beloesse Hukommelsen , men ogsaa at sve Dømmekraften, styrke Viljen og vcekke Samvittigheden , at lcere dem ikke blot at toenke rigtig , men ogsaa at handle ret og samvittighedsfuldt , i hvilken Livsstilling de end komme til at indtrcedc .

<h4>512</h4>Pragt og Ceremonier , Kulde og Afmaalthed ere her borte . Den ungdommelige Dnde af Naturens oprindelige Renhed , Sandhed og > / ) mhed troeder atter for os , aaben og nkunstlet . Vi indaandc en renere og friere Luft . Vi bevceges paa engang af en dybere Følelse af et scerdeles Himmelens Forsyn og en virkelig og fand Kjcorlighed paa Jorden . Vi fole , at der kan sindes noget faadant fom en barnlig og cedcl Tillid baade til Gud og Mennesker , og at det er muligt at handle i Overensstemmelse med denne Tro paa Guds Ledsagelse og Menneskers Redelighed . Forinden vi betragte den <b>Loerdom</b> , som kan vindes af Rebekkas Opforfel som MgtcfMe og <b>Moder</b> , ville vi for nogle blive staacnde ved hende med fcerligt Henfyn til den Begivenhed , der gjorde hende til dette , nemlig hendes Oiftcrmaal . De Omstændigheder , fom dermed vare forenede , vare eiendommclige for disse Tider og frembyde en Simpelhed , hvortil , som jeg allerede har bemerket , dcu nyere Historie ci kan opvise et Sidestykke . En Ting maa dog bemerkes , hun giftede sig med sine paarprendes Vidende og Samtykke . At hendes Fader Belhuel med Taushed forbigaaes , kan blot forklares faaledcs , at han ikke mere levede . Men Laban raadspurgtes . Her vare ingen Hemmeligheder med i Spillet , og desuden var det et Gistermaal, ved hvilket Religionens Fordringer til cu <b>vis</b> Grad havdcs for Olle . Hvis Rebekka viste altfor stor Iver med at indtrcodc i Mgtcskab , hvis hun altfor hastig fattede sin Beslutning , vil jeg vist ikke billige eller anbefale uoget saadant . Jeg har dog allerede sritaget hcndc for denne Befkyldning . Det kan vcerc naturligt nok at foretrcekke den gifte fremfor den ugifte Stand , naar Anledning tilbyder sig til at indtrcede deri . Man bor dog ei lade sit Hjerte ncerc en altfor utilborlig og urolig Lcengsel efter at saa sine ønsker virkeliggjorte . Man bpr aldeles ikke lade Indbildningen saa frit Spillerum i denne vigtige Sag . Tillad dig ci at behandle den med Letsind , <b>vis</b> ei en Opforsel , som knnde forraade , at du var mere begjerlig cftcr at indtrcede i Mgtcstanden end at blive ret slikket for dens Pligter . Giv ei Plads for den Antagelse , at du ci

<h4>523</h4>Tilfredsstillelse af sin Vegjerlighcd efter en <b>vis</b> Slags Mad og hans Partiskhed for sin SM Esau . Rebekka , hvis Hengivenhed altid var vangen , nidt ' scer og virksom for sin Andling , hMte den Opfordring , som hendes Mand gav Esau , og strar udkastede hun en Plan til at vende Velsignelsen til den anden , og som hun vidste , udvalgte EM . Hvad havde vceret hendes Pligt ? At gjMe Isak sagtmodige Forestillinger om det utilbørlige i at haudle i fuldkommen Strid med Guds aabenbarede Rand og Beslutning . En saadan Paamindelse til et sagtmodigt og gudfrygtigt Sind , fom hans utvivlfomt var trods fin Svaghed , vilde efter al Sandsynlighed have havt god Fremgang . Istcdet for dette lagde hun for Dagen , hvad der altid er blevet anfeet for at hMe til <b>Kvindens</b> Svagheder , en TilbMlighed til at tage fin Tilflugt til List , Kunster og Kneb , et efter heller at naa sit <b>Maal</b> paa krogede Omveie end med aabent og rent Spil . Det er overflMigt at gaa iud paa Enkelthederne i hendes Plan , dens hurtige Udførelse og dens Held . Fortællingen er sørgelig . Intet andet end Svig og Bedrageri , uucerdige Forklædninger , stsaalet Navn , foregivet Vildt , falfte Svar . Altsammen var daarligt undtagen Hensigten , som dermed tilsigtedes , hvilken dog ei kunde forandre Handlingen felv eller gjpre det onde ti ! godt . Det var en Vancere for Rebekka , en grufom Svig mod Ifak og den allergroveste Forbrydelse mod hendes SMs moralske Karakter . Vi maa ikke bebyrde Jakob med mere af denne Handlings Skam , end som virkelig tilkommer ham . Han var ikke den ftrste i Overtrcedelscn . Hans Følelser oprMte sig

<h4>536</h4>Anstrengelse , men endog under en fuldkommen Hvile . Det naturlige er ikke mindre skjMt i Kunsten end i Naturen , og Kunstens Fuldendelse er just det selv at forsvinde , idet den afbilder det naturlige i Natureu . Dette gjelder ogsaa om Adfcerd og Opfprfel ; der iscer er Affektation afskyelig og modbydelig . En ftgt og tilkunstlet Adfcerd , vcere sig , at det gjelder <b>Kundskaber</b> eller Dyd , Konversation eller blot endog Udtale , Klcedcr eller Sedvancr , er altid vederstyggelig. Den kan ei vinde nogen Agtelse eller Beundring , den maa tun voekke Spot . Maaske dette er en af de fornemste Daarstaber , mod hvilke <b>Kvinder</b> , og isoer unge <b>Kvinder</b> af Opdragelse , bM vcere van sin Post . Et affekteret Boesen vil sige meget ; det indeholder nemlig noget usedeligt og vanhelligt , det hentyder Paa List , Tvetydighed og forstilte Dyder , som ikke findes . I sin bestemte Form er det Hykleri, den afskyeligste Last paa Jorden . Nu har jeg dog egentlig for Aie en affekteret Adfcerd ; et hyklerisk Skin af Overlegenhed i et eller andet , en tillcert Rolle for at lyfe eller blcende . Dette udspringer sedvanlig fra den Forfængelighed, som til alle Tider er bleven anseet for at vcere en af Kvindekønnets Svagheder , og hvis Tilsted evcerelse forstyrrer mangen ellers clskvcerdig Personlighed . Forfoengeligheden er for <b>Kvinden</b> , hvad ærgjerrigheden er for Manden , og stjpnt den kan vcere mindre farlig , er den dog en smudsigere og hoesligere Feil ; det er en Form af Egen- ligesom avindsyg som umcettelig . Intet kan vcere mere stridende mod Evangeliets <b>Aand</b> , og den eneste Beskyttelse derimod er en dyb Hjertets Ydmyghed . Den maa I ifpre eder , I unge ; det er den Dragt , som bedst passer og pryder eder ; og yvor sinde I den andetsteds end i Evangeliets rige Forraadskammer ? Se hvilken elskværdig Venlighed hos denne unge <b>Kvinde</b> . Her er en fremmed , en Tjener , skjpnt « åbenbart ingen ringe Herres Tjener ; og dog hvor agtelsesfuld , hvor mild , hvor venlig var ei hendes Tiltale . Hoflighed og Venlighed er en Ynde , som Pryder begge Kjon , men iscer det <b>kvindelige</b> , medens derimod Racched er en hceslig Plet , iscer Paa en <b>Kvinde</b> . En raa <b>Kvinde</b> er noget , hvorfra

<h4>544</h4>og Paatagen Vlyhed med Blufærdighed . Du lever ei blandt asiatisk Tyrauui , Vankundighed og Fornedrelse , hvor <b>Kvinden</b> blot forener sig med Mandens sanselige Begærlighed, og hvor en grum Skinsyge udelukker hende fra Samfund med alle undtagen sine Medslavinder og deres foelles Tyranner . Du er fM i et oplyst Land , en oplyst Tid , og du har lige Del i alle Samfundslivets Glceder og Fordele . Benyt dig i denne Henseende af dine Rettigheder , bibehold din Plads , og naar du undviger Dristighed " , undvig da ogsaa at gaa for vidt i den modsatte Retning . I et foregaaende Kapitel antydede jeg , at den overdrevne Blufærdighed , fom ei eugang drister sig til at tale med det andet KjM , ei altid er af det reneste Slags . Der staar endnu tilbage nogle Lcerdomme , som en <b>Kvinde</b> har at hente af Rebekkas Opførsel under sit senere Liv , Lcerdomme at medbringe og ihukomme under hele Livet . Den ftrste er en almindeligere : forandrede Omstændigheder medfsre ofte forandret Karakter og forandret Opftrsel . Hvor forskjellig var ikke her Jomfruen i Mesopotamien og <b>Husmoderen</b> i Kana ' an ! Og er det vel noget usedvnnligt ogsaa nu at se Forandringer , langt mere omfattende og gjennemgribende , ftlge paa en <b>Kvindes</b> Indtrcedelse i Mgtestanden ? Betragt videre denne enkelte <b>Lcerdom</b> , at vi aldrig arbeide for et godt <b>Formaal</b> med slette Midler , eller med andre Ord , ikke det onde forat deraf maa folge det gode . Afsky denne Icsuiternes Grundsætning og 3 ) ndlingssats , at „ Hensigten helliger Midlet , " og afsky i Særdeleshed at folge denne ugudelige Grundsætning i det , som angaar Religionen . Hvilke Forbrydelser ere ei i Religionens misbrugte Navn blevne udMedc af romerste Ivrere for deres Kirkes bedste ! Og ogsaa af andre , i Navnet kristne , med renere Tro og besjeledc af en mildere <b>Aand</b> , hvor meget er ei blevet gjort , som det hedte , for Religionen , men i Virkeligheden kun for at befordre fceregne Hensigter , i Strid med alt , som Guds Lov og Kjcerlighed til Ncesten byder . Religionen vil ei betjenes af nogen anden <b>Aand</b> og bevægende Kraft end sin egen ; den foragter alle Tilbud ,

<h4>550</h4>Der ligger i Menneskets Vcefen en naturlig lighed for Smykker og Prydelser . Hvilken Forklaring man end vil give af felve Smagen , voere sig som en medftdt Opfatning af det skMne eller som en Vegjerlighed efter at opvcekke Beundring , Sagen er alligevel uimodsigelig . Man ser , at det er Tilfceldet saavel hos de raa som de dannede Folk ; det « åbenbarer sig paa samme Maade hos dem alle i Opmerksomhed paa sine Personer og sine Boliger og med et Ord i alle Samfundssedvaner og Skikke , Muslingerne , Fiskebenene eller Dyrtcenderne omkring den vilde <b>Kvindes</b> Hals saavelsom Diamanterne og Rubinerne i Dronningens Krone vidne om samme Tilb / sielighed . Denne Smag , skjMdt den i mange Tilfcelde kan voere ganske forkjert i Henseende til sit <b>Maal</b> , urigtig i sin <b>Aand</b> eller overdreven i sine Itringer , er dog i sit Boesen blot en Overensstemmelse med Guds egen Handlemaade , der „ satte sin Pragt paa Himmelen " og fyldte Jorden med SkjFnhed. Hvo kan vel overskue nogen af disse Naturens elskelige Scener og betragte Udviklingen og Fremstillingen af al denne Zirlighed i Former , denne SkjMhed og Afuerling i Farver , hos Blomsterne paa Marken , hos Fuglene

<h4>561</h4>Jord , jordisk ; Apostelen taler i Guds Kraft og lader os ftle den usynlige Verdens Magt . Medens Plato leder <b>Kvinden</b> til udelukkende at ftge den Mre , som kommer fra Mennesker , lcerer Petrus hende at ftge Guds , den alene fuldkomnes , den alene herliges Mre og Pris . Forinden jeg forlader denne Gjenstand , vil jeg just paapege den skjMne Maade , hvorpaa Apostelen henviser til den Del af vort Boesen , som det burde v « re vor fornemste Omsorg at pryde : „ Det skal vcere , " siger han , „ Hjertets skjulte Menneske i en sagtmodig og stille Aands uforkrænkelige Vcesen , hvilket er meget kosteligt for Gud . " Hvor stjsn er dog ikke denne Fremstilling , hvilket Dyb as Tanker ligger ei i disse faa Ord ! Det er Sjoelens og ikke Legemets Prydelse , som isoer bpr loegge Beslag paa den kristne <b>Kvindes</b> Opmerksomhed . Hvilke ere de Prydelser , som anstaa Sjcelen ? Sjcelcn er udødelig og uforgjengelig; saaledcs ogsaa dens Prydelser voere . Hvorledes kunne Guld og Juveler passe hertil ? Kan vel Diamanten oplyse eller overstraale Fornuftens Lys ? Eller Rubiner fortMe Hjertets Inde ? Eller den cegte Perle indfattes i Samvittigheden ? Eller en straalende Dragt kloede Karakteren ? Eller Fjcer og Blomster vaie over en ny og himmelsk Natur ? Nei , Sjcelens Prydelser ere Sandhed , Hellighed , Tro , Haab , Kjcerlighed , Ydmyghed , <b>Visdom</b> , Gloede , Sagtmodighed og Mildhed ; disse ere de Juveler , hvormed det indvortes Menneske burde Pryde sig . Det udvortes , det jordiske er forgjoengeligt . StFv er det , StM stal det atter blive . Hin SkjMheo , som nu straaler i Juveler og Mdelstene , en Prydelse for Valsalen og en Gjenstand for Avind , vil inden kort voere en Masfc af Forraadnelse , altfor gruopvcekkende til at beskue , siden et vederstyggeligt Skelet , en Samling af Benknogler , en Haandfuld Stpv , og hvor er da Sjcelen den udsdelige <b>Aand</b> ? Mon den stal bcere Legemets aflagte Pynt ? O nei , de have maaste endnu undgaaet Forgængeligheden , dog blot for at pryde andre til Dpden hjemfaldne Legemer , men ei til at berede Sjcele for evig Herlighed . Men tcenk nu paa den <b>Kvinde</b> , som , uden at lcegge altfor stor Voegt Paa

<h4>563</h4>Legemets Udstyrelse , forst og fremst efterstræbte Sjcelcns Prydelser ; se hende , som var iftrt Retfærdighedens Dragt og Salighedens Klcedebon , prydet med en ydmyg og stille <b>Aand</b> . Ogsaa hun og dei <b>Maal</b> af jordisk hun havde , hjemfalder , ogsaa det for en Tid til Forgængeligheden; men hendes udødelige og uforgjengelige <b>Aand</b> , over hvilken DMen ci har nogen Magt , fremtræder ei uden Prydelse for den eviges Ansigt ; thi de Juveler , hvormed den sFgte at Pryde sig paa Jorden , ere ligesaa uforgjcengelige som den er selv , og de fremtrcede med den for at straale i Guds Aine ; thi fra ham udgik deres Glans , til ham vender den atter tilbage . „ Menneskene anse det usselt og nedværdigende at vcere ydmyg . Intet er mere udsat for Foragt end en sagtmodig <b>Aand</b> , den er Daarstab for Mennester ; men hvad betyder det : mere end al Vancere gjelder vel det , at den for Gud er bFjt agtet . Og Tingene ere jo just saadanne , som han vurderer dem , og ei anderledes . Lad vcere , at det ei er Landets Sedvane , det er dog Sedvanen ved Hoffet , ja , det er felve Kongens , vor Konges egen Sedvane: „ Lcerer af mig , " siger han , „ thi jeg er fagtmodig og ydmyg af Hjertet . " Man ved , at de , som ere opdragne ved Hoffet , pleie at forfkaffe sig sine <b>Moder</b> derfra , selv om de ei bo der ; Vanderne kunne synes , at de se besynderlig ud i sine Dragter , de agte det dog ikke , men antage, at Hoffets Dragt er den bedste og smukkeste . GjFr ogsaa du sankedes , bryd dig ei om , hvad Verden siger ; du skal jo ikke forblive her saalcenge . Skaf dig baade Tol og Snit fra Himlens Hof . ydmyghedens Dragt og Sagtmodighedens Klcedning skal da sendes dig . Og boer disse Klceder for hans Skyld , som sender dig dem . Han har sin Lyst i at se dig dem iftrte ; er dette ei nok ? Vi kunne i intet komme til det rette , forinden vi have lcert os at foragte Menneskers Omdomme og blot rette vort Blik paa Guds Velbehag . " ( Leighton ) Men jeg gaar over fra den kristne <b>Kvindes</b> legemlige Prydelser til dem , som angaa hendes Tro og Betjendelse , og dette er ogsaa den egentlige Gjenstand for dette Kapitel.

<h4>582</h4>Prydelse for sin Menighed og bidrage meget til at indgive sine Herrer Tiltro til Sandheden . Ogsaa have isoer de fremfor andre rig og gunstig Anledning til med Pdmyghed, Mildhed , og Taalmodighed at Pryde Kristi Lcere . Paa den anden Side er det sørgeligt , at mange , som have havt <b>aandelig</b> sindede Tjenere i sit Hus , senere have havt Grund til at sige , at de aldrig mere Mske sig dem , thi de erc i enkelte Tilscelde ei bedre , og i nogle blot vcerre end andre . 3. Der findes videre adskillige Forstandens Prydelser, hvilke I bM efterstræbe baade for deres egen og for Religionens Skyld . Vistnok er det übestrideligt , at en sand og virkelig Gudsfrygt er sin egen Prydelse og faar ei Vcrrd ved noget ydre . ' Men fordi der findes Personer, fom ncere Fordom mod Religionen og have urigtige Anskuelser augaaeude dens Vcescn , ligesom om den skulde v « re stridende mod <b>Dannelsens</b> og Kundskabens Goder og den Inde , som smykker Karakteren , vilde det veere godt at bringe dem ud af sin Vildfarelse og ved Kundskaber, <b>Dannelse</b> og Anstand overtyde dem om , at Gudsfrygt aldeles ikke , som de tcenke , er blot et andet Navn paa Dumhed , Vankundighed og Raahed . Vearbeid saavel for deres Skyld fom for din egen din Forstand ved Studier. Vedligehold en brcendende ester <b>Kundskab</b> . Elsk Lcesning , men dannende , sorcedlende Lcesning . Gjendriv det Vanrygte , at <b>Piger</b> blot synes om Romaner , Kjcerlighedshistorier og <b>kvindelige</b> Eventyr . Scet Pris paa <b>Kundskaber</b> , ftg at finde Sandheden i alt , vcer ivrig i at randfage i Natnrens Undere og begjerlig efter at berige dit Kundstabsforraad med de Skatte , fom vor Tidsalder faa rigelig har bragt for Dagen . Men M ogsaa dine øvrige Evner ; stjcerp din Dømmekraft , overvei og tcenk efter . Der findes desvcerre mangen nng Perfon , hvis Hukommelse er belcesset med Navne og Aarstad men som er bedrøvelig klodset i sit OmdMme . Hun taler maaste Fransk , opramser lange Sider af Historien og glimrer med andre Talenter , men hnn er altfor indstrcenket til at danne sig en egen Mening eller til at fremfare Grunde for sine Tanker . Vi kunne naturligvis ei vente os , at

<h4>584</h4>alle <b>Kvinder</b> skulle vcere dybe Tcenkere eller spidsfindige HaarklFvere , men den tarveligste Opdragelse , hvis den anlcegges ret , bM kunne lcere <b>Kvinden</b> som et tcenkende Boesen at danne sig en i nogen Orad selvstcendig og bestemt Meniug . Lad det vcere saa , at dit Kjendstab til deu <b>Visdom</b> ikke leder dig til at foragte den lavere , paa det at ikke de , som blot ere dygtige deri , maa som FKge hvad de se hos dig , foragte det , som er HMst . det aabenbart , at de , som ere Guds VMn , ere mest begjerlige efter at lcere at kjende alle sin himmelske Faders Gjerninger, Skabningens Undere ikke undtagne . Overtyd disse Naturens Dyrkere om , at medens du ilsomt gaar dem forbi og i det allerhelligste ved Naadens Trone dyrker ham , som regjerer over alt skabt , dn ogsaa kan med dem tilbede ham i Forgaarden af hans Tempel og med dem opstige til den menneskelige Videns Hvider. Men , at naar de som Fugle med svagere Vinger og dunklere Blik her stanse og scenke sig nedad , saa kan du som Arnen med skarpere Aie uhindret svceve opad og sortscette din himmelske Flugt , hceve dig lige til de guddommelige , salige Regioner histoppe ! Hid lMer ogscm en <b>vis</b> Smag eller Opfattelse af og Kjcerlighed til alt skjomt og oplMet i Naturen , i Literaturen, i Dannelsen , i Anstanden , ja , ogsaa i den hellige Skrift , thi Vibelen er fuld af Exempler derpaa . Med en ren og forcedlet Smag vil du ogsaa nyde mere as Livets som kom ned sra Himmelen , af Guds Ord . Inspirationens <b>Aand</b> har prydet dets Sider med Skønheder, som undgaa de Ame , der ei ved <b>Uddannelse</b> ere blevne skjcerpede . Skriften er et Paradis , fom er rigt paa Blomster at beundre saavelsom paa Frugter at nyde . En <b>vis</b> Smag , i hvad man fedvanlig kalder Talenter eller <b>Dannelse</b> , er prydende for Religionen , naar den ei drives til yderlighed . Jeg har allerede forhen bemertet, at faadanne Ting , som Wre til <b>Kundskab</b> og Dannelse, ei ere at foragte . Vel indrammes , at det er en Synd at optage Time efter Time hver dyrebar Dag af Livet blot med de moderne tidsforFdende Bestjceftigelser >

<h4>640</h4>dramme altid om en Anledning til at glimre og vcekke Opsigt ; andre atter kunne ei blive forsynlige nok og bestrcebe sig blot for at spare saa meget som muligt . Paa alle disse Mander kan en <b>Kvinde</b> plage sig selv og fylde sit Sind med unådig og selvpaalagt Bekymring . Til alle saadanne sige vi : „ Martha , Martha ! du bekymrer dig og forstyrres ved mange Ting . " Foren derfor Marias brcendende Gudsfrygt med de bedre Egenskaber hos Martha , hendes Dygtighed og Flid i Husholdningen . Skulde du ville give dig tilfreds med de Egenskaber , som blot dig duelig for den Bolig , hvorfra du inden en Time kan vcere kaldet heden ? Voer ligesaa omhyggelig som Martha i de huslige Sysler , men vcer derhos ligesaa „ brcendende i Aanden , tjenende Herren" ( Rom . 12 , 11 ) , som Maria var . at forene alle en helligs Dyder med alt det , som anstaar en Mgtefcelle, <b>Moder</b> og <b>Husmoder</b> . Vcer alt , hvad du skal vcere , i dit eget Hus og alt , hvad du bsr vcere , i Guds Hus , det er , ide Ting , som tilhore ham . Opfyld en dig maaste bestemt Mgtefcellcs Forhaabninger ; kom hans ønsker Dette bcsr du , detmaa du . Hvad Kristus Msi ' er og begjerer , er at se dig sidde ved hans Fpdder og bFre hans Ord . Kom ogsaa dette » 2 nske imFde thi ogsaa dette er din übetingede Pligt . O I , som trykkes af Bekymring og Omtanke for disse mange Ting , lader der blive en Ende paa eders Sorg . „ Vcerer ikke bekymrede for noget , men i alle Ting lader eders Begjeringer fremføres for Gud i Paakaldelse og BM med Taksigelse ' . " En sand Gudsfrygts <b>Aand</b> stal mildne det huslige Livs Sorger , lette Bekymringen og stjcenke en Glcede , som vil scette eder istand til frimodig at forrette de huslige Ojcrninger , og en omhyggelig Agtpaagivenhed paa Hjemmets Pligter vil , langtfra at eder uskikkede for Omgangen med Gud , tvcrtimod give eder noget at bede om r tilstrækkeligt <b>Maal</b> og blive for eders Sjcel en tvingende Anledning til at ssge Herrens Ansigt . Og nu lader mig til Slutning gjpre eder opmerksomme paa den Beskrivelse af den sande Gudsfrygt , fom

<h4>679</h4>Ligesom Forældrene ei maa vcere Bornenes Tyranner, maa de ei heller blive deres Slaver . Et ynkeligt Syn er det at se et Hus , hvor alt synes at vcere OprM , hvor Ordenen er borte , den faderlige Myndighed brudt , Moderen foragtet og vanartige Bsrn i Besiddelse af Magten. Og dog maa en Fader saavel som en <b>Moder</b> adlydes, og det er hendes egen Feil , hvis hun ei er det . En <b>vis</b> og fast Styrelse , hvor Kjcerlighedens Haand holder Teilen , vil sikkerlig frembringe Underdanighed til Slutning . Men det maa vcere en Blanding af Kjcerlighed , <b>Visdom</b> og Myndighed . Underdanigheden maa kjendes af Barnet og erkjendes som en Pligt og ikke blot vindes som en Hengivenhedens Tribut . Myndigheden maa ei henstivne til Haardhed , ei heller Kjcerligheden til blotte Kjcertegn . Befalingerne adlydes , ei blot fordi det er en Tilfredsstillelse, men fordi det er en Pligt . En forstandig <b>Moder</b> anvender megen Takt ved Behandlingen af sine forfkjellige Sindelag . Thi inden en og samme Familie sindes ligesaa mange forfkjellige Gemytter, fom der sindes Søskende , ere aldeles lige i Sjcelsanlceg og Sindsbestaffenhed. En er dristig og ncergcmende , en anden stille og indesluttet ; den ene behøver at holdes i TMme , den anden at opmuntres . Den ene kan lettere ledes ved Mildhed og Lifter , den anden behover Strenghed . Ethvert Barn burde vcere Gjenstand for et scerstilt Studium . Kvccksalveri duger ligesaa lidt i Opdragelsen som i Medicinen. En og samme Behandling passer ei for alle Bprn , ligesaa lidt som en og samme Fode eller Lcegemiddcl passer for alle . Den <b>Kvinde</b> , som ei kjender alle sine Bsrns Egenheder og ei derefter retter sin Behandling af dem . er

<h4>696</h4>Og dog , tidligere end dette kan ske , er Plads for Moderens Indflydelse . En af de Drste Feiltagelser , som dog er san sedvanlig blandt Mo ' dre , er , at de fyrste to eller tre Aar af Barnets Liv ei ere af nogen Betydning for dets Opdragelse . Og dog er det sande Forhold det , at de ere de allervigtigste . Af de Indtryk , som modtages , de Grnndsandheder , som indstjcerpes , og de Vaner , som erhverves i de Aar , kan Barnets Karakter bestemmes baade for Tid og Evighed . Og klart er det , at Barnet har fedelig <b>Bevidsthed</b> , forinden det kan tale . Samvittigheden eller Evnen til at gjMe Forstjel mellem godt og ondt ved en <b>vis</b> <b>Moder</b> kunne udvikles hos Barnet strar efter , om ei fM dets anden Fødselsdag . Allerede da kan giFres begribeligt for det , hvad dets <b>Moder</b> anfer for ret eller uret , hvad der behager eller ikke behager hende . Vi HM MMe ofte sige , at deres ere endnu for unge til at lcere at adlyde . Men den <b>Moder</b> , som følger den Grundsætning , at Barnet stal kunne raade sig selv nogle Aar eller idetmindste nogle Maaneder , vil ril sin egen Skade finde , at den <b>Lcerdom</b> idetmindste ei saa let tages af Barnet . Moralst eller sedelig Opdragelse kan og bM gaa foran den , som blot stjoenkes Forstanden . Med Hjertet og Samvittigheden skulde Begyndelsen gjeres , og Grunden lcegges til Opdragelsen ; dette vilde betydelig lette alt videre Arbeide bande for Barnet felv og for dem , til hvilke dets Veiledning er anbetroet .

<h4>723</h4>Interessen forpges , nåar man faar at vide , at det er et Verk af en <b>kvindelig</b> Hacmd . Det er Villedet af en god Mgtefoelle , saaledes som det er tegnet af en <b>Moder</b> til Veiledning for hendes SM ved Valget af en Ledsagerinde gjennem Livet . Det „ Lemuels , en Konges , Ord , en fyndig Tale , hvormed hans <b>Moder</b> underviste ham . " Hvo denne Konge var , er usikkert ; mange antage ham for Salomo . Hele Stykket er et med Kunst udarbeidet Digt paa to og tyve Vers , hvilke ligesom i nogle af Salmerne begynde med Bogstaverne i det hebraiske Alfabet i deres Ordensftlge . Hvo Lemuel end kan have vceret , havde han faaet en <b>vis</b> og udmerket <b>Moder</b> . De advarende og undervisende Vers , med hvilke Kapitlet begynder, forfattede af denne fortreffelige <b>Kvinde</b> til Veiledning for hendes SM i Ungdommens Vaars og Forhaabningcrnes Tid , ere uforlignelige saavel med Henfyn til Indhold fom til Udftrelfe . Istedet for at meddele Negler af verdslig Klogskab og statsstyrende <b>Visdom</b> , indstroenker den udmerkede Forfatterinde fig med den sandeste og reneste Kunst til en Lovprisning af og Opfordring til Maadeholdets, Velviljens og Barmhjertighedens oedlere Dyder og til en nolagtig og uforlignelig Skildring af den <b>kvindelige</b> Dyd , som sikrest skulde befordre hendes SMs Lykke i det « ægteskabelige Liv . Hvilket udmerket MMster for er ikke at finde i denne Kongemoder ! Vi maa vel tomke os . at hun i sin Fremstilling gav et Billede af sig selv . Hvilte Sanner vilde vi ei faa at se , hvis alle vare velsignede med saadanne MMe !

<h4>772</h4>Beroligelse for min egen Samvittighed . Jeg har rost , hvor Ros fortjentes , og dette var ofte , men min Lov er ei nedsunken til Smiger ; og jeg har dadlet , hvor Dadel behøvedes, men jeg har gjort det uden Bitterhed . Mit <b>Maal</b> har vceret at befordre begge Kjpns Lytte ved at hceve og forcedle det af dem , hvorpaa begges Lykke for en saa vcefentlig Del beror , og at arbeide for Forøgelsen af Samfundets Velbefindende ved at renfc dets jordiske Kilde . Hvorvidt jeg heri har vceret heldig , er for mig umuligt nogensinde at kunne vide ; og i Mangel af Vished maa jeg triste mig med Haabet . Jeg har betragtet <b>Kvinden</b> i hendes Forhold til tvende Verdener , til Tiden og Evigheden , saaledes som hun er knyttet til denne ved en Mgtefcelles , Moders og Husmoders s , sde og hellige Pligter og til den tilkommende ved Udødelighedens endnu sterkere og varigere Baand , og derfor ikke blot Pligtig til at lcegge Vind paa de Dyder , fom tilhore dette Liv , men ogfaa de , fom berøre hendes egen Person i dens Forhold til Gud , Himmelen og Evigheden . Jeg har betragtet eder , mine unge Venner , som Hustruer , og Husmødre for en kommende Slegt , og min Bestrcebelse har vceret ret at berede eder for de Pligter , som fplge med et faa ansvarsfuldt Kald . Det har vceret mit <b>Maal</b> at aabne og berede eder en jevn Sti gjennem denne Jords Arken , at borttage fra eders Vei faa mange Stene og der at Plante faa mange Blomster , jeg kunde . Og om end min Bestrcebelfe har vceret ufuldkommen og min Opfatning mangelfuld , er jeg dog <b>vis</b> paa , at I , hvis mine Rand lcegges paa Hjerte og mine Vink fælges , om end megen Kummer endnu staar tilbage for eder , ei ville favne en rig Andel af TrM og Glcede . Det vil vcere eders og ikke min Feil , hvis eders Liv bliver et tomt Ark , en Arken uden nogen grpnncnde Plet , en Vildmark uden nogen belivende Kilde . Men jeg har skuet hen over denne Verden til det Liv , hvor I skulle befinde eder , fra al jordisk Amheds Baand , staaende alene med eders egen Personlighed og Udødelighed . Jeg er fuldt forvisset derom , og gid , at I alle vare saaledes , at I , enhver af eder ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112200022'target=_>, 1870, Bibelen, eller Den hellige Skrift</a><h4>1153</h4>ren mod hans Bud og Befaling ; Legemet overladcs ganske og aldeles til en i Opror mod Herren levende Person . Foreningen mellem Kjsnnene er af Herren indstiftet som et Afbillede af det Samfund , der sinder Sted mellem ham og dem , der hsrer ham til , Mand og <b>Kvinde</b> stulle vorde Et ester Aanden , men ogsaa i den fuldtomneste legemlige Forming , der stal vcere en Aabenbaring og Berer af den aandelige ; de stulle i den Grad vorde en <b>Aand</b> , at de endog blive et Kjsd . I denne Forening ligger nu et Afbillede af Samfundet med Herren , Christo , som videre udvikles Eph , 5 , 22. flg . Hele Menigheden staaer i et egtestabeligt , » : aandelig.legemligt Samfund med Christo ; derfor er vott Legeme hans Eiendom , hvilken vi alene have Ret at bruge ester hans Villie ; den Enkelte er et aandelig.legemligt Lem af Herrens Legeme , og det er alene i Wgteskabets uoplsselige <b>aandelig</b> » legemlige Forening , hvorigjennem Natur , drifterne ordnes og helliges for det hsiere <b>Formaal</b> , at frembringe og opfostre Arvinger til Guds Rige , og hvorved alene Herrens Samfund med sin Menighed paa Jorden sindbilledlig fremstilles , — at han wr blive Et med en Person af det andet Kjsn . 3. V . 14. Det er alene den ydre Skikkelse , hvori Legemet fremtrceder , (hvortil Bugen etc. hsrer ) , som er noget Forgjcengeligt, ikke selve Legemets ; Legemet er nsdvendigt for <b>Sjelen</b> til dens fuldstendige Tilværelse og skal i sin Tid gjengives den som et aandeligt , forklaret Legeme . Derpaa have vi et Underpant i Christi Legemes Opvekkelse , ved hvilken vi have Forvisningen ogsaa om vort Legemes Opvcekkelse . 4. V . 15. Eftersom Christus er vort Hoved , saa ere vi til Legeme og Sjcel Lemmer paa hans aandelige Legeme ; vi blive ved Troen Et med Christo . han skjenker den Troende sin <b>Aand</b> , men Leqemet stal igjen paa sin Si ^ e tjene <b>Sjelen</b> og saaledes ogsaa vere Christo helliget ; fslgelig maae vi ikke ved Horeri hengive det til en Anden .  Merk her vel , at det aandelige Samfund , vi have med Christo, ikke blot er et <b>Sjelens</b> , men ogsaa strekker sig til Legemet , at vi ere Kjed af hans Kjsd og Veen af haus Veen ( Eph . 5 , 30. ) ; thi ellers vilde Opstandelsens Haab vere uvist , ellers var vor Forening

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015010648113'target=_>Luther, Martin, 1847, Kraft- og saftfulde Kjerne af de Evangeliske Sandheder, udtagen af Dr. Martin Luthers Kirke- og Huus-postiller, indeholdende en fuldkommen Forklaring over alle Søn- og Fest-Dages Evangelier</a><h4>7560</h4>De derimod paa den anden Side , hvilke Prcrdike- Embedet ei er anbefalet , men som behove Undervisning og <b>Laerdom</b> , de , siger jeg , stal vcere Sonner , og lade sig <b>vise</b> , fore , erncere og pleie , og stille sig an mod deres Lcercre , som et fromt Barn mod sin <b>Moder</b> . Kjcrrligheden er vel ei saa stor i Born til Moderen , som i Moderen til Born , efter det almindelige Ordsprog : Kjcrrligheden gaaer mere nedad , end opad ; saa driver dog Naturen fromme Born til at crre og tjene deres Forcrldre i alle Ting . Naar det saaledes gaaer til imellem <b>Moder</b> og Esn , Prcedikantere og Lcerere , saa gaaer det vel til .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092203019'target=_>Holberg, Ludvig, 1861, Holbergs Comedier</a><h4>2864</h4>Det er underligt slags Folk disse Poeter . Om en anden crrlig Mand giver et Bresigen udi Tanker , trcrkker man ham til Daare- Kisten ; Men hos dem er saadant Tegn til <b>Lcerdom</b> og Dygtighed . Ellers sik jeg det Vers for got Kiob . De Folk maa have nogen Frihed , efterdi de ere saa lidet interesserede; Thi hvad er dog 12 si . for saadan lcerd Mand ? Jeg forstaaer ikke Verset , men kand dog see , at det er altid sine Penge vcerd . Nu skal jeg bilde min Hosbond ind , at jeg har givet 2 Rirdlr . derfor , saa vinder jeg ved dette Wrende 11 Mark og 4 § . Jeg haver kun liden <b>vis</b> Lon , maa derfor ncrre mig redelig ved Commissioner . Mangen brav Mand foder <b>Kone</b> og Born derved udi Kisbenhavn; Thi hvoraf skulde de gode Folk ellers leve ? Hvis jeg kunde flattere , som lesper gisr for min Herre , kunde jeg have deSmautzerogdeForcrringer som han faaer . Jeg skulde ikke roese mig selv ; Men jeg er en crrlig Karl , der ikke kand sige andet end jeg meener . Naar min Hosbond begynder at tale om sin Manddom i den Vrabandske Krig , tier Peer gandsse still ; Thi jeg veed , han har aldrig vcrret paa den anden Siide af Belt , end sige udi Brabant , som ligger nogle tusinde Miile herfra . Ney , Wrlighed varer dog lcrngst ; Det heeder hos mig , som Hollcrnderne suger : Thue das recht nicht , und sky den Tellffel . Kunde jeg saa vel Tydsk , som jeg kand Hollandsk , var Peer anden Mand end han er ; Thi jeg haver tit hort Nleiander Skolemester siige : Hvo der kand Tydsk , og hader nogle Midler dertil , kand reyse heele Europia igiennem . Men Fanden er udi dette DEN , DU , DNS , det kand man alleene fast i 14 Dage ikke lcrre .

<h4>8641</h4>pcrnille . Lad eders Fcimulus gaae til Side , medens vi tale sammen . petronius . Hvis det ikte var en Doctor , saa vilde jeg svcrrge paa , at det var Leonores Pernille ; thi de ere hinanden saa liige , som de kunde verre stsbte i een Form . Cosmoligoreuo . Hor , Petroni ! gak til side , saa lcrnge jeg taler med denne Herre . Jo meer jeg beskuer ham , jo meer han ligner hende . Pernilic . Lad ham gaae reent bort ; thi jeg har andet af Nigtighed at tale om . Cosmoligorcus . Gak kun Hiem til dine Forretninger , Petroni ! til jeg kalder paa dig . f ? erron , uo . Ach Herre ! jeg har og Lyst at tale med denne fremmede ; han ligner een af mine Kiendinger . pcriiille . Gaaerbort , naar jer Herre befaler jer . Sagte . Gid du faae en u-lykke med dine Kiendinger ! IHovt . ) Hvad har Sanl at giore blant Propheter ? Coomoligorl-uo . Gak strar bort , som jeg siger . < ) vill Lllul inter ? lapketaB ? Pctlonius sagte . De sige : hvad harSaul at giore blant Propheter ? jeg kunde jo svare med en <b>vis</b> <b>Dame</b> : han ledte efter sin Herres Mster . Ha , ha , ha ! <b>Maalet</b> er ogsaa , gid jeg faae Skam ! Pernilles . Ha , ha , ha !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008112403025'target=_>, 1861, Bibelen, eller Den hellige Skrift</a><h4>1317</h4>nodvendige for Mennestets legemlige Liv i hans nuvcerende jordiske Tilstand , oq ikke bore uoploselig sammen med Legemets eget Vcesen . Anderledes er det derimod med det , der angaaer Legemet som et Heelt . Ved Horeri anvendes ikke blot enkelte Lemmer as Legemet paa en Maade , der ikke sva » rer til Naturen , men hele Legemct und . drages Herren mod hans Bud og Vt . saling ; Legemet overladcs ganske og aldeles til en i Oprsr mod Herren le » vende Person . Foreningen mellemKjsn » nene er as Herren indstiftet som et Af < billede as det Samfund , der sinder Sted mellem bam og dem . der hore bam til . Mand og <b>Kvinde</b> stulle vorde Et ester Aandcn , mm ogsaa i den fuld < komneste legemlige Forening , der stal vare en Aabenbaring og Bcerer as den aandelige ; de stulle i den Grad vorde en <b>Aand</b> , at de endog blive et Kiod . I denne Forening ligger nu et Afbil < lede as Samfundct med Herren , Christo, som videre udvikles Efth . 5. 22. flg . Hele Meniaheden staaer i et agtcstabeligt, » : aandelig-legcmligt Sam < fund med Christo ; derfor er vort Le < gemc hans Eiendom , hvilken vi alene have Ret at bruge eftcr hans Villic ; den Enkelte er et aandelig-leaemligt Lem af Herrens Legeme , og det er alene i lEgtestabets uoploselige aandelig.le < gemligc Forening , bvorigjennem Natur » drifterne ordnes og helliges for det hoierc <b>Formaal</b> , at frembringe og op » fostre Arvinger til Guds Rige , og hvorved alene Herrens Samfund med sm Mcnighed paa Jorden sindbilledlig fremstilles , — at han tsr blive Et med en Person af det andet Kjsn . 3. V . 14. Det er alene den yd « Skikkelse , hvori Legemet fremtrccder ( hvortil Bugen etc. horer ) , som er no » qet Forgjangeligt . ikke selve Legemets Vasen ; Legemet er nodvendigt for Sja > len til dens fuldstcrndige Tilværelse og stal i sin Tid gjcngives den sometaan < deligt , forklaret Legeme . Derpaa have vi et Underpant i Cbristi LrgemeS Op ' vcekkelse , ved hvilken vi have Forvisnlngen ogsaa om vort Legemcs Opvcrkkelse .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092303053'target=_>Arndt, Johann, 1867, Johann Arnd's Sande Christendom</a><h4>4425</h4>ftlgelse — Tro og Haab er saaledes afbildet i den hellige Skrift , at Troen strax antager det , som om det var sagt om den alene , og det troende Menneske ligesom seer sit eget Hjerte i de Helliges Exempler , ligeledes sit Kors , dertil ogsaa sin Trpst , sin Hjcelp og sin Frelse . Deraf kommer det , at Troen grid er det gamle og det nye Testamentes Twstesvrog , tilegner sig dem og ftrer sig dem til Nytte , saa at deraf- visselig bliver en levende Glcede og Twst i Hjertet . Hvorledes skulde da ikke Guds Ord leve i det menneskelige Hjerte ? Er da Guds <b>Aand</b> i Ordet d » o og livlss ? Ere ikke Herrens Ord <b>Aand</b> og Liv ? ( Joh . 6. ) Eller mener du , at det udenfor dit Hjerte uden Troen vil bevise og ovfylde sin Kraft ? la , Guds Ord bliver daglig opfyldt , baade paa og i de Troende og paa og i de Ugudelige : hine til Trost , Beskjermelse , Seier , Frelse og til Salighed , men disse til Hevn og Straf , og naar man vil aabne Vinene , skal det beftndes saaledes . — Se her , om dette er weigeliansk . * ) Hvor ofte minder ikke den Helligaand os i vort Hjerte om et Trsstesvrog , i hvilket vi fornemme megen Viisdom , <b>Lcerdom</b> og Trost . Hvad er denne Propheti : Jeg vil skrive min Lov i Eders Hjerter ( ler . 31. ) ? Hvad tykkes dig , var det ikke en HK Guds Gave , da Herren ovloo Skriften for Apostlerne ? Men du har lcenge cedt den med Hud og Håar . Hvad mener du , er det ikke HM fornodent, at Gud Herren siger til dit formprkede Hjerte : Bliv Lys ? Hvad seer du i Abels Offer , i Syndfloden , i Noahs Rogoffer , i Arken , i de rene og urene Dyr , i Duen og i Ravnen ? Har du ikke ogsaa opbyggetßabelsTaarn i dig ? Kjender du ogsaa Melchisedek ? Have Abrahams Gjcester ogsaa nogensinde vceret hos dig og holdt Maaltid med dig ? Har du ogsaa nogensinde tcenkt paa Loths <b>Hustru</b> ? Er du Såras eller Hagars Son ? Har du ogsaa modtaget den aandelige Omskjcerelse ? Har du ogsaa erfaret Abrahams Fristelse med hans egen Son ? Veed du ogsaa Forskjellen mellem lakobs og Esaus Velsignelse ? Er Esau ogsaa i dig Jakob fiendsk ? Har

<h4>5667</h4>forenet det med Din evige og guddommelige Natur , og harovfyldtDitLegemes Tempel med Din guddommelige Herlighed ; dette er min Meste Glede og Trost , og der kan aldrig vederfares et christent Menneske sterre Glede , end at Gud har Menneskene saa kjer , at Han selv er bleven Menneske . Ak hvor kjer har Du , o Gud , Menneskene , at Du gM os til Guds Slegt , og deelagtige i Din guddommelige Natur ! Derfor hedder Du , o kjere Herre , lefus Christus ! Immanuel , det er , Gud med os , og vor eneste Midler imellem Gud og Menneskene . Og paa det at vi altid stuld e kunne troste os ved Dig , har Du givet Dig et saa huldsaligt Navn , at Dn hedder Jesus Christus , vor Frelser og SaliggMer , en Salvet , en Konge og Ippersteprest ; og har ladet os Alle kalde ester Dit Navn Christne , det er . Herrens Salvede ; thi Du har salvet os med Din Helligaand til et aandeligt og kongeligt Prestedpmme , og til det tilkommende evige Rige , til Konger og Prester , hvilket tilkommende Rige vi i Troen vente paa . Derfor er Du , o Herre Jesus Christus ! undfangen i Modersliv af den Helligaand , paa det at Du efter Din hellige menneskelige Natur kunde blive salvet over alt <b>Maal</b> med den Helligaand og med alle Aandens Gaver , paa det at vi Alle kunne modtage af Din Fylde . Ogsaa er Du derfor undfangen af den Helligaand , paa det at Du skulde vere den Allerhelligste , uden al Synd , og vi Alle skulde blive helliggjorte i Dig , og den menneskelige Natur i Dig atter blive renset , fornyet , velsignet , forlsst fra Forbandelsen , og opfyldt med den Helligaand . ODu allerhelligste Undfangelse , som har bragt os Velsignelsen tilbage ! Derfor er Du ogsaa fM af Jomfru Maria , imod og over al Natur , paa det at Du kunde blive et sandt Menneske og et Menneskes Som , paa det at Du kunde gMe os til Guds Bsrn ; paa det at Du kunde blive Jomfruens Ssn , den forjettede <b>Kvindens</b> Sed , Abrahams Sed , Davids SM , det rette Davids Skud , den deilige Kvist af Ifai Rod , paa hvem Herrens <b>Aand</b> hviler , forn paa den rette yppersteprest , der skulde forsone os med Gud ved at offre Sit hellige Legeme og Blod , som Du har

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012102924021'target=_>Holberg, Ludvig, 1867, Holbergs Comedier</a><h4>2967</h4>Det er underligt flags Folk disse Poeter . Om en anden crrlig Mand giver et Orefigen udi Tanker , Ircrkter man ham til Daare- Kisten ; Men hos dem er saadant Tegn til <b>Lcerdom</b> og Dygtighed . Ellers fik jeg det Vers for got Kisb . De Folk maa have nogen Frihed , efterdi de ere saa lidet interesserede; Thi hvad er dog 12 tz . for saadan lcrrd Mand ? Jeg forstaaer ikke Verset , men land dog see , at bet er altid sine Penge vcrrd . Nu skal jeg bilde min Hosbond ind , at jeg har givet 2 Rirdlr . derfor , faa vinder jeg ved dette LGrende 11 Mark og 4 p. Jeg haver kun liden <b>vis</b> Lon , maa derfor ncrre mig redelig ved Commissioner . Mangen brav Mand foder <b>Kone</b> og Vsrn derved udi Kisbenhavn; Thi hvoraf skulde de gode Folk ellers leve ? Hvis jeg kunde flattere , som Jesper gisr for min Herre , kunde jeg have de Smautzer og de Fordringer som han faaer . Jeg skulde ikke roese mig selv ; Men jeg er en crrlig Karl , der ikke tand sige andet end jeg meener . Naar min Hosbond begynder at tale om sin Manddom i den Vrnbandske Krig , tier Peer gandske still ; Thi jeg veed , han har aldrig vcrret paa den anden Siide nf Belt , end sige udi Brabant , fom ligger nogle tusinde Miile herfra . Ney , Mrlighed varer dog lcrnqst ; Det heeder hos mig , som Hollcrnderne suger : Thue das recht nicht , und sky den Teuffel . Kunde jeg saa vel Tydsk , som jeg land Hollandsk , var Peer anden Mand end han er ; Thi jeg haver tit hsrt Alercmder Skolemester siige : Hvo der kand Tydsk , og haver nogle Midler dertil , tand reyse heele Europia igiennem . Men Fanden er udi dette DER , DU , DAS , det tand man alleene fast i 14 Dage ikte lcrre .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015062608033'target=_>Rosenius, C.O., 1879, Hemmeligheder i Lov og Evangelium</a><h4>362</h4>siger han utvetydigt : Fader , jeg har givet dem den Herlighed , som Du har givet mig . Og deunc Herlighed er hans Lyst . Hans Lyst er hos Menncskcns Bsrn . Men ligesom vor Herre Jesus har sin storste Lyst i sine egne fuldkommenheder , saa har han itkc mindre Velbchagelighed i sine egne Gjerninger . „ Herren har Behag i sine Gjerninger " . dæmoner David . Enhver pleier jo at elske , hvad han selv har hvor meget mere da Herren , hvis Gjerninger alle erc idel og Fuldkommenhed . Tcenk vel paa denne Omstcrndighed ; tyt den har en vcrldig Trost med sig . Skulde Gud ikke vcrre tilfreds med , hvad han selv har gjort ? Han maa jo have Behag i sine egne Gjerninger ! men hvor erc nu hans Gjerninger ? Mon hos i Helvede ? Bevare os Gud ! Han er selv sinVoerkmester til sin ncervcerende Skikkelse . Mon hos Englene i Himlen da ? Endnu ikke ! De have selv i Gjerningspagten forhvervet sig deres Herlighed og tilkcempet sig deres Kroner . Mon da hos de selvretfærdige og i disses egne Fortjenester og Dyder ? Aldeles ikke ! det er deres egne Gjerninger ; de skulle ogfaa selv svare for sig . Hvor finde vi da Herrens Gjerninger , som cre hans Lyst ? Der hvor en solder slaaer sig for sit Bryst : „ Gud vocre mig arme Tynder naadig ! " Der hvor en Bartimceus raaber ved " Veien : . . Jesu , Du Davids Son , forbarm Dig over mig ! " Der hvor en lanllnceisi <b>Kvinde</b> taler om Hundene og Smulerne , og en sonderknust Magdalena bader Jesu Fodder med sine Taarer . Hvor en Paulus glad udbryder : „ Mig er Barmhjertighed vederfaren " , og en Asaf : „ Herre , naar jeg har Dig , sporger jeg hverken efter Himmel eller Jord " . Hvor en Simon Petrus begynder at boeve i Stemmen for at udsige : „ Herre , Du veed alle Ting , Du veed , at jeg elsker Dig " . See , hvor saadanne Ting foregaa , hvor Stenhjcrter blive forvandlede til et blodt Ler i hans Haand og de staalhaarde Ansigter til Vox , saa at han kan aftrykke sit Billede deri ; hvor Sjoele , som aldrig have spurgt efter Herren , nu begynde at torste efter den levende ' Gud som Hjorten efter det stiske Vand . hvor de selvretfærdige begynde at kurre som Duer pall Ruinerne af deres Rctfccrdighedsbygning . og de <b>vise</b> begynde at agte deres <b>Visdom</b> for Daarskab , hvor arme Syndere begynde at groede ved hans Fodder , og fordomte Misgjerningsmoend gladelig vove at paaklllde ham som deres Forsvarer imod Anklageren der , der er hans Gjerningcr , og hvor hans Gjerning er , der er hans Lyst . Saa er dll hans iblandt Mcnneskens Born , og det iblandt hans fattige Syndere . Hos dem er hans Fornoielse , bos dem hans Oines Fryd . Hvad nyder ikke en Naturven af en skjon Egn . og den . som har plantet en Vingaard , hvor frydes ban ikke ved at beskue dens Blomstring og Frugtbarhed ! Saaledes har Herren sin Lyst i den Urtegaard . som han ved sit Forsoningsblod og sin <b>Aand</b> har plantet paa Jorden . Den nye Skabning i cn Synder , hvor fryder den ikke hans Oie ! Han kan ikke see ' sig derpaa ; thi den er hans egen Skabning / Derfor siger han 9

<h4>402</h4>( V . 16. ) . Desforuden kan man tcrnke sig . hvilken Helligdom denne Enkes Bolig var formedelst den store GM . hun herbergerede . Hendes Hus var nu et Allerhelligste i Landet , hvor den store Profet Elias boede og gjorde Mirakler . Israel havde mistet ham , Enken i Sarepta havde ham ham , som kort derefter foer til Himlen med brcrudende Heste og Vogn . Nu boede han i Stilhed , skjult hos denne Enke , medens Herrens Vrede over Israel sagte stred til sit <b>Maal</b> . Hvilket Lys , hvilken Naade maa ikke have udbredt sig ' af „ Elias ' Aano og Kraft " ( Luk . 1 , 17. ) i den lille Hytte i Sarepta ! Man tcrnke sig , hvilke Lcrrdomme den hellige Mand gav sin Vcrrtinde , og hvilke Bsnner der opstege i hendes Hus ! Og hvor maa hun ikke have frydet sig ved med sine rorende Erfaringer af Guds Naade og Hjcrlp og med det aabenbare og fortsatte Mirakel stadig for Die at kunne sidde alle Dage ved hans Fodder , som havde vcrret Guds Mund til at udtale Tilsagnet derom , alle Dage hore den fromme og <b>vise</b> Gudsmand at tale om himmelske Ting ! „ Af Hjertets Overflødighed taler Munden " . Han , som siden blev sendt tilligemed Moses for at tale med Kristus paa Forklarelsens Bjerg „ om hans Bortgang , som han skulde fuldbyrde i Jerusalem " , han har sikkert her i Hytten talet om den kommende Messias , lcrst og forklaret Stykker af Skriften derom samt nu og da istemt en Lovsang til Herren . Englene have visseliqen sect deres Lyst paa denne lille Kirke i Orkcnen . Saa fredeligt og skjent , saa stille og acmdeligt , saa helligt og hoitideligt var det hos Enken i Sarepta . Men hvad skeer ? Uformodet viser sig over hendes Bolig en liden Sky den voxer den bliver mortere den bryder los i en Styrteregn en <b>mor</b> ! Bedrovelsens Sky . Og vi skulle snart finde , hvilke uformodede og vigtige Opdagelser om hendes Sjcrls inderste , der fremkaldtes herved . Hendes Son bliver syg og doer hendes eneste , elskede Barn , der nu var saa meget dyrebarere for hendes Hjerte , som de nu ved den underfulde Velsignelse vare befriede fra Hungersdoden . Han begynder at jamre sig . hans Tilstand forvcerres med hver Stund Sygdommen bliver betcenkelig , haard , farlig den arme Moders Nod stiger til det yderste og efter nogle Dage er den lykkeligste Bolig forvandlet til et Sorgens og Taarernes Hjem . Moderens eneste Gloede ligger nedslagen som et blegt og koldt Lig paa Baaren . Da ingen Ting , hverken Sorg eller Gloede , skeer uden Herrens Befaling , da alle vore Hovedhaar er talte , og ikke et af dem falder tit Jorden uden vor himmelske Faders Villie saa sporges : Hvad mente Herren med denne haarde Hjemsogelse i denne fromme og lykkelige Bolig ? Hvorfor borttog han Enkens eneste Son ? Af Folgerne mcrrter man Herrens Hensigt . Det var en stor og viatia forandring , som herved skulde frembringes i Enkens Sjcel . 10 *

<h4>429</h4>Mnr ! hans magelsse <b>Visdom</b> ! Herren har nemlig til det stolte , egensindige og indbildske Menneskes til alle Tider sat ham paa saadanne Prover , at Herrens Ord og Menneskets Fornuft skulde komme i Strid , og at Menneskets Velfcerd skulde bero paa . om han sor Herrens Ords Skyld kunde foragte sin egen Fornuft . Derved har den store , alene <b>vise</b> Herre udrettet dette , at ingen stolt , stiv og egensindig kan som saadan tomme ind i Himmerige , og deri har den store Gud handlet ret fortræffeligt med os . Men ikke nok dermed , at man hengivent og lydigt maa underkaste sig Herrens Ord og foragte fin egen Fornuft og Tykke han har tillige stillet det saaledes , at ogsaa de , som ville antage Ordet og deri soge sig frem til Himmerige , skulle dog ikke Mde Veicn , om de ikte derhos boie sig for selve Ordets Herre og ydmygt bede om Noglen til den indelukkede Skat , bede om den Helligaand . Derom skriver Lnther : „ Da Gud gav os sit Ord , sagde han : Jeg stal vel lade det skrive og procdike tlart og tydeligt , men dog altid soie det saaledes , at det stal bero paa min <b>Aand</b> . hvem der stal fatte det . " O , Du prisverdige Guds <b>Visdom</b> ! Paa denne Maade har Du udelukket alle stolte , formastelige Sjccle fra Din Salighed , alle , som ikke ville ydmyge ug for Dig . Her see vi , hvad vor Herre Jesus solte , da han frydede sig i Aanden og sagde : „ Jeg priser Dig , Fader , Himlens og Jordens Herre ! at Du har skjult dette for de <b>vise</b> og forstandige og aabenbaret det for de umyndige . Ja , Fader ! thi de var saaledes behageligt for Dig . " Vi have i den hellige Skrift mange Prover paa denne Guds Handlcmaade , at han har villet knuse og sondcrbrhdc Fornuften . Saadanne cre f . Ex . Guds Bud om Isaks Offrmg , om Omskærelsen, om Kobberslangcn i Orkencn m . sl . Hvor maa ikke Abrahams Fornuft foruden hvad hans Hjerte folte have krympet og vriet sig ved Herrens Befaling om den eneste Sons Offring , saa den maatte sige : Det kan ikke voere Guds Bud ; han har jo lovet mig en talrig Slcegt i denne Son ofu . Hvor maa ikke Fornuften have studset og krympet sig med Hensyn til Omskcrrelsen ! Hvor maa den ikke have vriet sig hos de slangebidte i Orkenen , da det blev dem sagt , at det blotte Syn paa den ophoiede Kobberslangc skulde give Helbred . De maa jo have tcrnkt : Giften er jo allerede inde i vore Aarcr ; det kan dog ikke yclbredcs ved et Blik ofv . Et icerstilt lecrcrigt Excmpel lcese vi i 2 Kong . 5 Kap . om den syriske Hcrrforer Naaman . Denne havde en strockkelig Spedalskhed , som ingen i Landet kunde helbrede . Omsider Horte han ved sin <b>Tjenerinde</b> , at der fandtes i Israel en Profet , Elisa , som kunde helbrede den skrækkelige Sygdom , Spedalskhed . Naaman reiste nu til Elisa , holdt med sin Vogn udenfor Profetens Bolig og lod fremfare sit Wrinde . Men nu skulde hans Fornuft brydes . Profeten behagede nemlig ikke at gaa ud til den syge Hoerforcr , men sendte ham kun det Bud : „ Gak hen og to Dig syv Gange i Jordanen , saa stal Dit Kjso

<h4>535</h4>derved , at Djævelen tilligemed den drcebende Bcgjoerlighcd , han indsiydcr i Folclscn og Fantasien , ogsaa formorker Forstanden og overbeviser Sjcrlen om . at Synden itte cr Synd , idctmindste ilte farlig , men paa den og den Grund stal den kunne ansces som tilladelig , ja uskyldig og aldeles ikke farlig . Hertil kommer nu ogsaa de ugudeliges Raad , „ Djcovlelcrrdomme " , som tage Syuden i Forsvar , og samme Kristen , som for altid har gyst ved disse Lcerdomme . finder dem nu i Fristelsens Stund ganste gode , kloge og sande . Og paa dette Punkt er det fcedvanlig , at Faldet bcroer . Om Sandheden beholdes i sin Renhed , saa kan denne atter fore Sjcrlen tilrette ; men tabes Sandheden , og den arme SM antager Djcrvlclceren og begynder at forsvare Synden , saa er det forbi . Men der findes " andre Begærligheder , som endnu mere fint og uformørket borttage Livet . Disse cre de , som sigte til noget , d ' cr i sig selv cr usiyldiat og godt , men som bliver en Afgud for Hjertet og derved bliver til Dod . Dette skeer , saasnart Hjertet begynder ' at hcrngc puledes ved noget andet end Gud og hans Naade . at dette Andet bliver det forste Anliggende , optager den meste Omsorg , at dets Opnaaclsc er den storste Gloede og Fornoiclsc og dets Tab cr den storste Sorg . Da cr Folgerne aldeles de samme , om end Sagen i sig selv er nok saa uskyldig og god , som om det var den uhyggeligste Last . Det er en uskyldig Sag at spise Kjod , men ved et utoilet Begjcer efter Kjod , var det . at Israels Born . . forkastede Herren " ( 4 Mos . 11. 20. ) og bleve nedslagne af ham , hvorfor det Sted hedder . . Begærligheds Grave " ( v. 34. ) . Det cr en uskyldig Sag at have Jordegods eller Avlsssaard eller <b>Hustru</b> , men naar Kristus vildc angive de Hindringer , som afholde Sjcrle fra Guds Rige . saa noevncdc han disse ( Luk . 14 , 18 — 24. ) . At elske Fader og <b>Moder</b> cr ikke blot uskyldigt , men befalet : men „ den som elsker Fader eller <b>Moder</b> mere end mig " , siger Kristus , „ den lan itte vwre min Discipel " , kan ikke ind ' gaa i Guds Rige . Naar nu Mennesket gaacr i <b>aandclig</b> Ladhed' og Sikkerhed , hvorledes sknlde han da frygte for Vcgjcrringcr , som have saa nsiyldige Ting til <b>Formaal</b> ? Nei , da seer han blot paa Formaalcts Art og tcrnkcr : det cr jo intet ondt . det jeg soger : have ikke ogsaa de Hellige havt Jordegods , Handel eller en Mage ? skulde jeg i ' tte ogsaa turde soge saadant ? osv. men vil aldrig give Agt paa . ' om denne Sogen cr afgudist , cgenraadig og selvist . om Sagen indtager Sjcel og Hjerte saaledcs , at Gud , hans Naade og ' hans Velbehag cr derimod et Intet , blive for übetydelige til at kunne tilfredsstille og fornoic Sjcrlen ; nei . derpaa giver det arme Menneske nu ikke Agt ; han cr ikke oprigtig uok imod sig selv til at undersogc dette , men farer let over det og kan maastcc ved at lccse eller hore saadant tcrnke : Ja jeg har jo det og det . som jeg higer efter ; men saa kjlvrt som Gud og hans Naade yar jeg det dog ikke ; nei . jeg vil dog hellere blive salig osu . og agter nu ikke paa . hvorledes Hjertets Stilling egentlig cr , eller hvad Her cr hans egentlige Hjcrtcsprog . Omsider kommer det derhen .

<h4>610</h4>Saaledes taler Kristus om Menneskers Elendighed og Omvendelsen . Her bliver da intet andet Raad , ingen anden Hjoelv end en idelig og uendelig Forladelse , og hele Kristi Rige forbliver et Forlade ' lsesrige . Vi ville slntte med Luthers kostelige Ord i en Prcedilen over „ Fodtva-tningen " ( Joh . 13 , ) . Han siger : „ At to Fsdderne det kaldes ogsaa , at den ene stal forlade den anden hans Forseelser , og som Paulus kalder det : „ den ene slal bcrre den andens Oyrde . " Thi der bliver intet andet af ; Du sinder i hele Verden ingen Kristen saa ren , at han ikke har nogen scrrstilt Feil og Strobelighed ; den ene er heftig og vred ; den anden er sorgmodig og besynderlig tilsinds ; den tredie for munter eller undertiden letsindig ; den fjerde karrig og egennyttig ; den femte for gavmild og odscl . Summa : det er , som ogsaa Hedningene have sagt : enhver har sin Feil og Skrobelighed , som den anden har atbcerepaa . Du synes da vel , at om der ilke var en saadan Feil ved , ham , saa vilde Du gjerne omgaaes med ham ; men saa megen Skrøbelighed er Dig stor og utaalelig . Men hor , naar Du seer saadan Feil hos Din Husbonde , <b>Hustru</b> , Born . Tjenestefolk , Naboer og andre , saa stal Du tcenke paa , at det er Din Pligt at to deres Fodder ; gjor det ogsaa ; hold dem en saadan Skrobelighcd tilgode og sce til , hvorledes Du kan fore dem paa den rette Vei ; og betcrnk , som det i sandhed er , at Verden er intet andet end en uren Stald , hvor vi umulig kunne gaa uden at faa Foddcrne urene . Dette vederfares mig , det vederfares Dig og i en Sum : alle Mennesker . Hvad ville vi da gjore ? Skulle vi derfor lobe ud i Skove og Orkener og ikke have Omgang med nogen , ligesom Munkene forhen have gjort ? Nei , om Du stal to andres Fodder , saa maa Du ikke vcere alene , men midt iblandt Mennesker , som i Verden vade igjennem et urent og besmittet Sted . Der stal Du boie Dig ned , om Du end selv har rene , vakkre Fodder , og bringe dem Vand , Pjalter, Halm og Ho til at to og aftorre dem med , paa det ogsaa deres Fodder kunne blive rene . Dette skeer paa den Maade , at „ den ene blerer den andens Byrde , " det er , at den ene hjertelig antager sig den anden , har Medlidenhed med ham , ikke foragter ham , men rcekker ham Haanden, underviser ham venlig , raader og hjcelper ham , saa at ogsaa han kan forbedres . Om Du nu er forstandig , <b>vis</b> og lcerd , saa foragt ikke derfor de vankundige og uforstandige , hold Dig ikke bedre end de , men brug Din <b>Visdom</b> til , at ogsaa de maa blive <b>vise</b> og forstandige . Ligeledes om Du er from , kydft , tugtig , maadelig og bliver ikke let fortornet , men en anden har ikke saadan Gave , saa foragt ham ikke derfor , men antag Dig ham , sog at forbedre ham , hvormed Du kan , og toenk som saa : jeg har vel , Gud stee Lov , ikke denne Feil , men saa har jeg en anden , hvorfor jeg behover , at man ogsaa har Taalmodighed med mig . Derimod har rnin Nceste ogsaa en god Dyd ved sig , hvormed han kan tjene mig igjen , og

<h4>853</h4>Dette er Hemmeligheden af en Kristens egentlige Liv og af al sand Fred og Kraft , nemlig den fortrolige Barneomgang med Gud . „ En ret Lcere og en kristelig Vandel " , det er for mange alt , hvori deres Kristendom bestaacr ; men til saadanne stal Kristus endnu sige : Jeg kjendte Eder ikte ; vi omgikkes ikke ; gaaer bort ira mig ! Nei , til en ret Kristendom horer endnu noget andet som det aller vigtigste og aldeles nodvendige , nemlig at Du ogsaa naaer i Forening med Gno og i en fortrolig Omgang med ham . Tamt , naar Kristus selv , og det paa mange Steder , saasom i Math , 7 , 23. Mllth . 25 , 12. Luk . 13 , 25. 27. siger , at han skal saadanne , som have „ cedet og drukket med ham " , at han blot ikal sige : „ Icg kjendte Eder ikke " da maa det Ord have nogen hoiit betcenkelig Betydning ! Tet betyder ikke , at han ci har havt nogen <b>Kundskab</b> om dem : thi . han kjender jo alle Fugle , saa at ikke en af dem salder til Jorden uden hans Villie . Det maa da betyde , man ikke bar havt Omgang med ham , at man kun har vceret Tjener , men ikke havt det Djerte , at man har levet af hans Kjeerligyed , i sortrolig Omgang med ham , som Bruden med nn Brudgom , som Borncne med deres hulde Fader eller <b>Moder</b> Paulus uger ogsaa tvende Gonge i Rom . 8 , 15. og Gal . 4 6 en naar vi have saaet „ en sonlig Udkaarelses <b>Aand</b> " , saa rander denne : Abba , Fader ! Og „ dersom nogen ikke har Kristi <b>Aand</b> denne er itte hans " lNom . 8. 9. ) . Det er so da aldeles nødvendigt, at vi maa bave en saadan barnlig Udkaarelses <b>Aand</b> i Ålenet , at vi her i Tiden ove Omgang med vor Gud , om vi i Evigheden ikulle leve med ham . Den . som kan tage det til Hierte til Selvvrovelse , han tage det ! Men da er det ogsaa nodvendigt , at vi ret forståa og betcenke BeMnenhcden af den sande Omgang med Gud , at vi ikke forverle dermed disie selvgjorte Bonneovelser , hvormed saa mange bedrage ug ie ! v. Apouelcn nger : Eftersom I da ere Ssnner , 22

<h4>1202</h4>yave forjljelligt Blik i Evangeli Frihed . Paulus og de stcerke i Troen saa tydeligt , hvorledes Kristus var Loveus Ende , at naar han var kommen , behsvedes ikke „ Skyggen " ( Hbr . 10 1 ) De svagere i Troen havde vel ogsaa fundet deres Retfærdigheds Liv ug salighed i Kristus , men knnde dog ikke indsee , hvorfor den Lov , som Gud selv engang havde givet , ikte ogsaa burde iagttages ' hvorom snart noget mere . Den gamle , lovmoessige Vane hcrngte endnu ved deres Samvittigheder , og deres Blik var ikke helt klart forståa dette lettest ved at see paa det , som vi endnu daqliqen finde ide troendes Hob . De ere alle efter lamgere eller kortere HMd med Vantro og Egenrctfoerdighcd endelig komne iqiennem snevre Port til Evangeli Hemmelighed og have mndet deres eneste Frelse og Trost i Kristus : han alene er nu hele Gjenstanden for deres stærkere eller svagere Tro . deres mere hungrende eller mere tilfredsstillende og glade Samvittigheder ere de alle „ den fri <b>Kvindes</b> Born " og alle lige kicrre for Guds Nien hvor forstjclligt er ikte endnu Blikket hos ' de jvagc og de jtorrke ? Naar de fsrste sige : „ Da Du er kommen til Kristus , saa tam ! nu paa at <b>vise</b> Dig den store Naade vcerdiq " iaa uge destdstc : „ Har Du fundet Kristus , saa glced Diq i terren altid , og Kristi Kja-rlighed tvinger os til alt godt . " Naar de ° fsrste Uge : „ Gud er trofast , saa lcengc ui tro paa ham " , saa sige de sidste- vl blive utro , bliver han dog tro ; han kan ikke forncrqte sig ' elv Naar de forste tcentc : . Nu bsr jeg gjore godt . siden jeg har begyndt at tro " , jaa gjore de forstedet gode , forend de have tcenkt derpllll . ' Naar de forstc efter en begaaet Utroskab opgive deres , Mtmng og forst efter et vist <b>Maal</b> af Omvendelsessora ville gaa frem og bcgjcrrc Forladelse , saa begrcrde visselig de andre Uge Mvcl dereZ Feiltrin , men gjore deraf ingen Beredelse for Naadcn men ice al deres Trost i Kristus alene . Naar de forste yelft fordybe jig i . hvad de ere blevne ved Troen , eller hvad de Mille blive i jig selv . saa beskue de sidste sig helst i det Billede , Oe have udenfor stg i deres store Overhoved . Naar de forstc beflitte ug paa Bon . <b>aandelig</b> Tale m . m . ' efterdi en Kristen skal bor og maa dette , saa flyder hos de sidste Bsnnen og Talen af Hjertets fri Lyst og osv. Herved bor dog mcrrkes , at ingen er jaa jtoerk , at han ikke stundom bliver neddragen i Svaahed og Hkrobeiighed i Troen , saa at enhver Kristen ' ogsaa har Me forstMge af Svaghed og Styrke Nu siger Apostelen : Antager Eder den skrsoelwe i Troen hans Meninger , eller : betymre ftti VUttghedeu . Der er jandt . de ere uforstandige , disse svage : der cr noget jygeligt i deres Kristendom ; men just derfor bor man antage jig dem jaa meget desto ommerc : de forstandige bor da vleie og bcerc dem . ligejom man i et Hus aller mest pleier oa er om om det Barn . jvm cr svagt : „ Bekymrer ikke Samvittigheden " Uger v , voMen . veo at „ domme deres Meninger " eller misbruge 30

<h4>1229</h4>den . som tilhorer de Kristne , er den vcerste , er gjenstridig , , egen « sindig og ulydig eller om en Husfader eller <b>Husmoder</b> , som holder sig til de Kristnes Hob , er haard eller egennyttig lige overfor Tjenere og underordnede , saa at disse , om de end have et kristeligt Sind , dog hellere vilde tjene hos de ugudelige . O , hvor forstroekkeligt ! Du maatte dog hellere onsteDig at voere skjult nogle Alen under Jorden , end at Du skulde fremstille et saa vrangt Billede af Kristendommen ! See . om I dsntte Eder selv , fan bleve I itte dsmte , siger Apostelen . Dette gjcrldcr baade med Hensyn til Gud og Mennesker ; thi den , som selv erhender, begræder og afbeder sine Skrøbeligheder og alvorligt strceber at blive dem kvit , den har derved borttaget den Forargelse , som laa i dem ; men den , som ikte lader sig sige , „ ikke lader formene sig Synden " , men hylder , forsvarer og forscctlig udover den , ham stal Apostelen mode med den alvorlige Tiltale : Vil Du Vide , g Du forfængelige Menneske , at Troen uden Gjerninger er dsd ! Luther siger : „ Her gjcelder det at see itte pall deres Mund , men paa deres Levnet ; " her gjoelder det ikke at Du kan tale vakkert , men ogsaa om Du har nogen Nandens og Kraftens Vejvisning. Det er let at deromme sig af Gud og Kristus , men om Din Tro ogsaa er sand og oprigtig , det stal bevise sig derved , at ben ogsaa medforer en hellig <b>Aand</b> i Dit Hjerte , som angriber binder og korsfæster Dit gamle Menneske samt gjor Dig Ydmyg og kærlighedsfuld . Du stal dog visselig aldrig her i Tiden blive ti ' lfreds med Din Helliggjsrelse og Kraft , saa fremt Du er Vaagen og ikke forblindet af Forforcren i Selvhellighcdens Indbildninger Du stal dog altid fole mere af Din Hynd end af Din Hellighed; men andre stulle vidne , og i de lysere Tider stal Du ogsaa selv med Apostelen sande , at „ Guds Naade mod Dig har ikke vccrct forgjcrves " li Kor . 15. 10. ) at de mest fremstaaendc Aandcns Frugter virkelig ere fulgte af Din Tro , stjondt ikke i det <b>Maal</b> , som Du snstede . Vi gjentage atter her . hvad vi paa et andet Sted have sagt , at om Du har en saadan Helliggjorelsesiver . at Du forst og sidst taler om Helliggjsrelse og Gjerninger , og derved har Vinene mere rettede paa andre end paa Dig selv , saa at det egentlig er andres Mangler , der besucere Dig og ikke Dine egne . da er det et meget betccnkeligt Tegn paa . at Du er en forblindet og gjerningssyg Lovhelgen , som endnu ikke har taget Sagen synderlig alvorligt for egen Del og endnu ikke rigtig vced , hvad Apostelens Ord betyde : og jeg dsde ( Rom . 7 , 10. ) . Men om Du derimod er en saadan . som selv er tilintetgjort samt forst og sidst ivrer for Lammets Mre og er rentud " nidkjcrr imod alt. som ophcever sig derimod og vil gjoelde mere i Menneskers Dine end . det at Guds Son var lagt under Loven , kæmpede vor Hellighedskamp paa Jorden og tilfidst bloote og dsde paa Korset om Du er en saadan som forst og sidst ivrer for dette Lams Mre og derfor bespotter og forhaaner al vor egen Retfærdighed , naar den ml

<h4>1235</h4>eller han haabede , at Ilden skulde vedblive , ogfaa efter at han havde ophsrtmed at loegge Brande til ? Vi forståa jo let , at en saa daarlig Mand snart skulde blive skuffet i sin Forventning Men en endnu storre Skuffelse og Mangel <b>moder</b> den Sjoel , som tomter faaledes paa Helliggjorelfe og gode Gjcrninger , at den mister Troens Enfoldighed og Trost i Kristus , ikke mere har sin Ncrrmg r Kristi uforskyldte Naade , men enten neddrages i en sygelig Trceldomsaand og saaledes mister al Kraft ifolge Joh 15 , 4 eller ogsaa faacr en hemmelig Trost i sin egen Helliggjorelfe. lever as sit eget Arbeide , sin Strenghed , Alvor oq Fromheo. O , hvor ynkeligt er det ikle med de Sjcele ! De have deres « it og Skat i deres egen Trossab og Helliggjorelfe oq vide ltte , hvilket frygteligt Tegn den Trost er . „ For saadanne var det oeore " . siger Luther , ~ at de laa i Skarnet op over Drene " Hold derfor altid det Punkt rent og klart , at intet andet erHelliqgiorclfc end det , som flyder af den ydmygede og i Kristus be < naadede , salige <b>Aand</b> . flyder ud af Kristi Kjcerlighed som Saften af stammen . hvad Du gjor af lovmcessig Bevcrggrund , er Aandcns men „ Lovens Gjerninger " , og „ saa mange Wn holde sig til Lovens Gjerninger , cre under Forbandelse " ' Dig derfor , at Du ikte lader Samvittigheden fanges under Aag formedelst Dine mange og store Mangla O nei , da mllatte Du hellere ligesom Enhjormngen , om hvilken man uger . at han ikte tan fanges levende , bryde Dig med Kraft " Garnet og sige : Jeg ringeagter Me Guds Naade- thi om NetfVrdlssyeden kommer af Gjernwger . saa er Kristus dsd forgzwves . Hellere vil jeg ikke vide af en eneste god Gjcrmnq mo at jeg siulde lade den blive min Skat . min Retfærdighed oa ' den stor min Syndeflod , langt storre Lammets og kan Du blot beholde denne Trost , blive i KicerKghed , saa flyder deraf altid det rette Sind og Kraft iom Jesus sagdes Bliver i min KWrliqheo , siulle , 5 wre megen Frugt , og min Fader bliver fortzerliaet m undet . v , ville anmcerke . er . at det Lys . som I give , maa Lov og af Frelserens Exempel . ' Ligesom indtil Dodcn er Grundvolden for alle saa er og Frelserens Forskrifter og Exempel Regelen derfor . Gud vil et hoiere <b>Maal</b> af Helliggjorelfe N tor ncervoerende Tid have . og han har givet os Monstret derfor . Han har undervist os om det honeste Maa ! af Helligyed ; baade Gnds Lov og Jesu Exempel ere fuldkomne , saa a . fuldkommenhed er vort <b>Maal</b> oq intet mindre . Et lavere <b>Maal</b> end dette vove vi ikke at foreskrive hverken os selv eller Eder ; thi der fmdes intet lavere i den hellige Skrift . Men her er det aldeles noovendigt at betomke , at vor Frelser , hvis dyrebare Exempel vi skulle efterfolge , var ikke blot hellig udvortes men ogsaa indvortes i Hjerte , Tanker , Hensigter oq alle Sjcelens Evner ; derfor er det aldeles ikke nok , at vi blot Vaage over vore 31

<h4>1483</h4>dets Gudstjeneste . At deres Proester forvendte Loven ( Math , 5. ) og korsfæstede Kristus var ham ilke tjcert , kunde heller ikke bortjage ham : thi hvad gjorde ikke Paulus for Frelse ? „ Paa det jeg kan vinde dem " , at „ jeg dog for altmg kunde frelse nogle . " See . dette var hele hans Sag . Men naar han derimod var ude iblandt Hedningerne , da taug han aldeles stille om den levitiste Lov og brcendte af hellig Iver mod de falske Apostle , som bekymrede hans Galater med Lcercn om Omskærelsens Nødvendighed. Alt gik kun ud paa at beholde alle ved Enfoldigheden i Krifto og „ ikke gjsre nogen Forhindring " for det eneste saliggørende Ord om ' Omvendelse og Tro . See her , hvorfor han i Antiokien bestraffer Petrus for det samme , som han selv gjor i Jerusalem ! Da Petrus i Antiokien var omgiven af en Skare omvendte Hedninge og dog rettede sig efter nogle faa jsdiste Kristne , som kom fra ' Jakob i Jerusalem , hvor de troende endnu havde Nidkjcerhed for Loven , saa „ irettesatte Paulus ham for alles Oine . " See Gal . 2 , 11 — 12. Men naar Paulus tom til Jerusalem, hvor han blot havde loder omkring sig . da gik han selv i Templet og rensede sig m . m . SeeApg . 21 , 17 — 27. I Brevene til de Romere og Galater bestraffer Paulus , at de hcengte ved Omskærelsen : men naar han tommer til Lystra og finder den troende Discipel Thimoteus . „ hvis Fader var en Groeter " , hvad gjor han da ? Johan den samme Paulus omstår ham selv , Og hvorfor ? „ For Isdernes Skyld , som vare paa disse Steder ; thi alle tjendte hans Fader , at han var en Grader " lAvg . 16 , j . Saaledcs lader han sin klare Forstand om det rette her vige " for Kærlighedens Nidljcrched „ for at gjvre mange salige . " Hvor klart han end vidste , hvad der i sig selv var rettest , gjorde han dog kun det , som var det nyttigste , nemlig for ikke at bortstodc disse loder fra sin Prcediken . Saaledes gjor den sande Kjcerlighed og <b>Visdom</b> . Og om end nogen vilde heri see Dobbelthed , Mennestebehag og Hykleri og sige : Paulus vandrer ilte efter nogen <b>Vis</b> Regel , saa stal den hellige Apostel med Aandens Vidnesbyrd svare : „ Kristi Kjcerlighed tvinger mig saa ledes " „ paa det jeg maa vinde desto flere . " Og den , som i sand Tro vil eftcrfslge de nu anforte Exempler af Moses , Kristus og Paulus , maa troste sig paa samme Maade , om han faaer samme Beskyldning . Ogsaa iblandt os opstaa mange Meninger . Den ene bekymrer sig over Sabbaten , fordi den ikke holdes paa den forestrevne syvende Tag , paa hvilken „ Gud hvilede fra alle sine Gjerninger " , men paa den fsrfte . paa hvilken Kristus opstod — og han sporger : Hvor byder Guds Ord denne Forandring ? Den anden bekymrer sig over de smaa Borns Daab og sporger : Hvor er den befalet ? Den tredie bekymrer sig over , at <b>Kvinden</b> beder med übedcekket Hoved , eller at noaen spiser Blod osv. efterdi der ogsaa i det nye Testamente er talet derom ( 1 Kor . 11 , Apg . 15 , 20. ) . En fjerde

<h4>1576</h4>Tanker , at naar vi med den storste Andagt og Gloede have feiret Jul og Paaske , saa kunne vi strax fattes Tro paa de forste Bogstaver af Guds <b>Kundskab</b> . Vi tro saa meget , vi ville , naar blot ingen Provclse <b>moder</b> os , men saa snart noget ret bekymrende indtroeffer , da er det , som om der var ingen Gud til . Det er derfor , vi bor sor Alvor gjore os rede for . hvad vi virkelig tro om Gud , om vi tro , at han endnu lever iblandt os , og at han er , som han beskrives i Ordet , om han er dette til enhver Tid , ogsllll naar det bliver rigtig kulsort for vore Dine , at vi da kunne stille vort Hjerte tilfreds for ham og sige : Hndnu er Du , o Gud , lige Mr og har en lige saa uendelig Kjccrlighed , Troskab og Magt som det og det Aar , da jeg smagte og saa , hvor liflig Du er , saa at jeg kunde smelte derved . Jo mere bittert og morkt Du nu gjor det for mig , desto storre er den Naade og Gloede , Du derved bereder mig . See , til en faadan Guds <b>Kundskab</b> bor vi komme ; da have vi en hoi og vceldig Trost i alle mulige Bekymringer, en salig Ro i de bittreste Tider . Men her <b>moder</b> os en mere oplyst Anmcerkning , som siger : „ Det er sandt , alt , hvad der <b>moder</b> os , er os tilsendt af den trofaste Gud , men kun saa loenge det ikke angaaer , hvad der beroer paa Menneskets Frihed . Om jeg selv ved min Egenvillie og andre fyndige Bevæggrunde har fort mig i Ulykke , saa maa jeg tage Skylden paa mig selv og ikke troste mig ved Guds Forelse . " Svar : Dette er i sig selv rigtigt , men bliver ofte urigtig anvendt . Det er sandt , at Gud har givet os en <b>vis</b> betoenkelig Frihed , hvorved vr kunne gjore os ulykkelige og Evighed , nemlig naar vi vedblivende og forsEtlig modstaa hans <b>Aand</b> ; men her er saaledes tvende Ting at moerke : for det forste , at om Du end ved Synd og Daarstab har bragt Dig i Ulykke , men dog nu er kommet til Anger og en lydig <b>Aand</b> , saa at Du herefter vil folge Herrens Raad , saa er Guds Naade og faderlige Godhed saa stor , at han vil vende alt Dit onde til det gode , alle Din Daarstabs Folger til Din Tjeneste ; thi alle Ting maa tjene dem til gode , sgM dog NU elste Gud ( Rom . 6 , ) thi Gud vil ikke bestandig gaa i rette , eiheller beholde Vrede evindelig . Han er en saa naadefuld Gud og Fader , at stjondt man loenge har trodset ham , men engang omvender sig , er han os siden lige saa huld . - som om vi aldrig havde syndet . " Det var ved en stor Egenvillie , Israel fik Konger , men Herren overgav dem dog ikke ; yan gjorde dem lige saa meget godt siden ; kun maatte de lide mere . Paulus havde i sin Vantro vcerct en Kristi Fjende og Forfolger , men Kristus gjorde ham dog til den stsrste Apostel , ja gjorde endog to gode Ting af den store Synd , nemlig dels et mocgtigt Sonderknuselsesmiddel for Apostelen selv si Kor . 15. 9. ) og dels et mocgtigt Trostemiddel for andre ( 1 Tim . 1 , 13 , 16. ) . Her see vi , hvad nys blev sagt , at Gud ogsaa kan vende vore storste Feil og Daarskaber til vor Tjeneste , naar vi blot engang komme til Omvendelse. Ja Luther anmcrrker , at vor Herre Gud ogsaa maa

<h4>1702</h4>og Gudfrygtighed . Og Paulus siger , at denne „ daarlige Prædiken" om „ den korsfoestede Kristus " er en „ Guds Kraft " ( 1 Kor . 1 , ) . Og dette endnu eftertrykkeligere , naar han spsrger Galaterne paa Samvittigheden : Kun dette vilde jeg vide af Eder : var det ved Lovens Gjerninger , I annammede Aauden eller ved Troens Forkyndelse ? Mon da han , som meddeler Eder Aanden og udretter kraftige Gjerninger iblandt Eder , udretter det ved Lovens Gjerninger eller ved Troens Fortyndelse? ( Gal . 3 , ) . See , her er Feilen , hvorfor Du ikke har faaet et aandeligt Liv i Dit Hjerte , en Guds Kraft , men koemper og arbeider saa msisommeligt , medens alt er dsdt og afmoegtigt . Du tjender Ve Kristus det fattes Dig . at Kristus er ikke forklaret for Tig Dit Hjerte har ikke ved guddommelig Aabenbarelse forstaaet den Salighed i Kristus , hvorom Du har lcest , hort og talet . O , hvilket nyt Liv Du skulde saa i Dit Hjerte , hvilken overnaturlig Kraft , om Kristus blev forklaret for Din SM , saa at Du r Aandens Lys saa Dybet og Hoiden af hans Kjoerligheo , af hans Forsoning og Naade og blev aldeles frigjort fra Loven , aldeles salig saadan , som Du er ! See paa de rigtigt frigjorte , gjenfodte Sjocle , hvorledes alt , hvad der horer til Liv og Gudfrygtighed, kommer ligesom af sig selv i deres Hjerter ; de have en hjertelig Gloede over deres Frelser og en fortrolig Omgang med ham , de have en ny hellig Lyst til at voere ham tro og lydige i alt , de have en ny Kjoerlighed til alle Mennesker , saa at de kunne groede og bede paa deres Knoe for deres Omvendelse og tale de hjerteligste Ord til deres Voekkelse ; og med Brsorene t Kristo ere de saa forenede , at de have alt tilfoelles med dem daade glcrdeligt og bittert ligesom Lemmer i Legemet ; de have en ny Ydmyg <b>Aand</b> , saa at om de end formedelst en heftig og slem Natur kunne forsce sig , ere de dog boiede , saa at de tage Skylden paa stg , erkjende deres Feil og bede om Forladelse ; de have et nyt rent der Kder og pmes ligesom af Ild . naar de fsle Kwdets Urenhed osv. Hvorfra have de alt dette ? Ene og alene ved den levende <b>Kundskab</b> om Kristus og deres egen Benaadning . Der var en Mand , som i al sin Tid , ogsaa under sin opvakte oa sogende Tilstand aldrig havde havt Kraft til at sige et godt Ord om Gud ; men omsider fik han i Aandens Lys at see Jesus og sm egen fulde Benaadning i ham , og da blev han saaledes tilsinds, at han ikke kunde afholde sig fra at tale om Frelseren til alle som . om i hans Noerhed , idet han dreves af en usigelig Gloede og Kjoerlighed ; og da hans <b>Hustru</b> bekymredes herover , bad og formanede ham til at ophore dermed , svarede han : „ Om jeg blot havde faaet titusinde Kroner , saa havde det jo vccret vanskeligt at beholde den Lykke for sig selv ; tcenk da , naar iea nu har faaet hele Frelseren og er bleven saa frigjort fra al min Synd , at har saa meget deraf , som der kan ligge paa mit

<h4>1898</h4>men jeg avlede Oder i Krifto ved Evangelium ( i Kor . 4. 15. ) . Hvilken scerdeles Titel : „ Tugtemestere i Kristo ! " Han siger ikke Tugtemestere til Kristus , hmlket han kalder Loven ( Gal . 3 , 24. ) men Tugtemestere i Kristo , som altsaa tugte dem , der allerede ere i Kristus ; men Faderomheden kjende de ikke , det antyder Apostelen dermed , at han saa skarpt adstiller dem fra „ Fcedre " eller de , som have „ avlet os i Kristus ved Evangelium " , saa han siger : Om I end havde titusinde Tugtemestere r Kristo , have I dog ikle mange Foedre . Forskjellen imellem Tugtemestere og FVdre er den samme som imellem de to <b>Kvinder</b> , der tvistede om Barnet for Salomons Domstol ( 1 Kong . 3 , ) . „ Guds <b>Visdom</b> " lcerte Kongen , hvorledes han skulde forståa , hvilken der var den rette <b>Moder</b> ; han sagde : „ Deler det levende Barn i to Dele og giv hver af dem Halvdelen . " „ Ja deler det " , sagde den , som falskelig vilde tilvende sig Barnet ; men den virkelige <b>Moder</b> udraabte forstrækket: Nei giv hende det levende Barn ! „ Thi hendes inderste broendte for hendes Son " , siger Texten . Her er Hemmeligheden ; det beroer paa et vist Moderhjerte forFaarene , som siger : Hug ikke , stoer ilke saa übarmhjertigt ! Og dctte ikke blot fordi man er vm for den Lidelse , som foraarsages ; den betyder mindre ; men hor og mcerk engang : det er fordi Du kan fordoerve et Liv . Der er et levende Barn under al denne Skrobelighco , ncmr oi tale om dem , som fole og erkjende deres Synd og gjerne vilde vcrre fri for den ; der er et Liv derunder , som Du kan hugge ihjel , et Hjerte , som har begyndt at leve i den omtaalige Barnetillid til Herren . Paa denne Barnetillids Bevarelse bcroer forst selve Livet , det aandeligc og evige Liv . og derncest ogsaa den sande Helliggjorelse. Tilintetgøres Barnetilliden , da er ogsaa al sand Helliggjorelse tilintetgjort , og al Fromhed bliver siden blot Lovens ' Gjerninger . Og da har man mcd sin haardhcrndede Skceren kun udrettet saa meget som den Gartner , hvis svcede Frugttrce ikte voxte saa lige og vakkert , som han onstede , og som derfor star , bandt og afstivede det , iudtil det dode under hans Hcender . Det er vel sandt , at det havde vceret smukkere , om det var voxet lige , og en <b>vis</b> , forstandig Oftbinding og forsigtig Besteering havde vceret god og iscrr Borttagelsen af det , som forhindrede Frugtbarheden ( Joh . 15. 2. ) men vil man rette alting uden Indsigt og Forstand om , hvad Troeet kan taale , saa doer det og fortorres , og da var det jo dog bedre at have et kroget Troe , som dog havde Liv , end et meget lige , men dsdt . Derfor ere Luthers Ord her fulde af Hyrdeuisdom , naar han taler om , hvor omt og varsomt de svage maa behandles , og tilfoicr: „ de blive vel med Tiden stærkere . " Og paa et andet Sted Yttrer han : „ Man siger vel , at Oiet er en omtaalig Ting , men jeg siger , at en sm Samvittighed er det aller smtaaligste . Det steer overmande let og snart / at den bliver forstyrret , men det er itte saa let at hjeelpe den tilrette igjen . " Den , som har denne Folslss og <b>Visdom</b> , den kan verre en qoo Hvrdc ; men de « , som

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008090903023'target=_>Wallin, Johan Olof, 1848, Prædikener over de aarlige Søn- og Helligdages Evangelier</a><h4>835</h4>Persons Anseelse , her var ikke Isde eller Greker , Tråel eller Fri , Mand eller <b>Qvinde</b> ; Alle vare Eet i Christo Jesu , alle vare Guds Born ved Troen paa lesum Christum . Og Herren lagde daglig til Menigheden dem , som bleve salige , og den hele Mengde , som troede , var eet Hjerte og een <b>Sjel</b> , og Guds Naade var over dem Alle . Hvilken Glede gjennemstrsmmede Apostelens Bryst ved dette Syn , og udbredte sig derfra vederkvegende over de christelige Menigheder , som han havde forsamlet for Herren! Hvor lykkeligt , om ingen qvcelende eller kold Vind havde oprsrt den klare Strsm ! Men det kjsdelige Sind og den indbilske Halvoplysning begyndte allerede i hans Dage at maale og at veie det Evige med Tidens <b>Maal</b> og Vegt . Talen om Korset blev snart Isderne en Forargelse og Grekerne en Daarskab ; de Forste vare for sandsclige til at lade deres Hjerte oplsftes af Guds <b>Aand</b> , — de Sidste for stolte til at tåge Fornuften fangen under Troens Lydighed . De havde ladet sig dsbe , idet de vel anede , at Noget fattedes dem , som de behsvede, endskjsnt de ikke ret gjorde sig Rede derfor ; de bekjcndte Christum ; men det var en anden Christus , end Evangelisterne og Apostlerne havde prediket ; de kom stedse tilbage til de gamle Hverdagsbegrebcrs Spsrgsmaal : " Er ikke denne en Tsmmermands Son ? hedder ikke hans Fader Joseph og hans <b>Moder</b> Maria ? Hvad har vel han at byde os , som vi ikke selv ville , om vi finde forgodt ? Hvad har vel han at lere os , som vi ikke selv vide , om vi tenke efttr ? " — De lagde saaledes ind i Christendommcn, hvad de selv lystede , og borttoge , hvad der ikke passede dem , — og enten de sammensmeltede den med gamle Fordomme , eller beffare den efter nye Paafund , saa fandt de altid deres Tilhengere ; idet Menneskene , — da som nu ,

<h4>953</h4>hvis <b>Moder</b> , — da han sidste Gang gik op til Jerusalem for at scrtte Kronen , ikke paa sig selv , men paa sit Vcrrk , — med uforstandig Vmhed bad om en 2 Eresplads for hver af dem paa hans hoire og venstre Side i det Rige , man troede , han var kommen for at oprette som en verdslig Konge paa Jorden . Da de hertil svarede Ja , tilfoiede Frelseren : min Kalk skulle I vel drikke ; men det Ovrige beroede paa hoiere , i Himmelen fattede Beslutninger, som vare skjulte for mennestelige Sine . Han kunde endnu ikke fuldkommen haeve deres forudfattede Mening om Hensigten af hans Komme ; dertil var endda ikke Tiden kommen ; men han vilde itide lade dem forstaae , at hvilkensomhelst 3 od i timelig Henseende der blev dem til Deel , skulde de , som hans Navns troe Bekjendere , ei undgaae de haarde Provelser , gjenncm hvilke det alene vilde blive dem muligt at vorde hans Riges vcerdige og engang salige Medlemmer . Inden Tidens Grcrndscr kunde ei saa store Ting fuldbyrdes . Her skulde hans Krone blive af Torne , hans Throne det blodige Kors , og af de to , som der kom til at verre paa hans hoire og venstre Side , blev ikkun den Bodfcrrdige frelst . Lidelsen har saaledes i sig selv ingen Nogel til Himmeriges Rige . Ikkun da , naar dens rensende og oploftende Kraft griber vor <b>Sjel</b> , og forer os did , hvor Gud vil , forer os i Jesu hellige Fodspor til Bonnen , Hcngivenheden , Taalmodigheden , Selvfornegtelsen og det seirende Haab , ikkun d a stal den bittre Kalk , som Ingen undgaaer , blive os gavnlig . Den iskjamkes paa ulige Maade , men i tilstrekkelig : <b>Maal</b> for Alle , som paa Livets dunkle Stier strcrbe frem til det <b>Maal</b> , der er forelagt . Det er saaledes hverken Sporgsmaal om , i Forveien at fremkalde den , eller , naar Tiden er kommen , at undgaae den , men ikkun at lade den tjene til sit rette Oiemed , og blive saa nyttig , som den havde til Hensigt at blive . Kunne vi saaledes modtage , saaledes tomme den ? Og hvorledes stulle vi kunne det ? Det er , hvad vi fremfor alt Andet behove og spsrge om , — det er , hvad Du alene kan lcrre os , som selv dråk den bittreste Kalk , o vor Forsoner og vort Monster ! Ak , naar Du , den eneste Hellige , maatte

<h4>1049</h4>Godes stille Udsvelse at samle Dig varigere , mere tilfredsstillende Skatte end disse Forgjamgelighedens , der , ligesom falske Venner , forst bedrage Dig og saa forlade Dig ; ja , loer af selve Mangelen , selve Savnet ret at vurdere endog Forsynets timelige Naadegaver og taknemmelig at modtage dem , naar de igjen , om end i sparsommere <b>Maal</b> , blive Dig til Deel . Og skulde de ei blive Dig til Deel , ganste i Forhold til det , som er nyttigt for dit sande Vel ? Du veed ei , hvorledes det skal gaae til ; men han veed det , som har de ukjendte Midler i sin Haand , og leder sine Born undcrligen , men sikkert til <b>Maalet</b> . Lider Du som en Christen, saa vcer <b>vis</b> paa , at Du fsr eller senere bliver forsonet med Lidelsen . Sorgen er da Glcedens <b>Moder</b> , og paa Modgangens steenbundne Mark opskyder lidt efter lidt den Blomst , hvis Krone ei udvikles paa Jorden , men hvis Knop allerede der hedder Salighed ; thi Saligheden , min Ven ! ligger ikke i det sandselige Menneskes Nydelse af det Ncervcerende , som , endog naar det er paa det Bodste , altid er indskrcenket , afbrudr og usikkert , nei ! den ligger meget mere i en Christens fromme Forventning om , at Alt vil blive bedre , i samme Forhold som han selv bliver bedre , og at Alt vil blive lysere , jo mere Lyset oprinder i hans egen <b>Sjel</b> , jo klarere han indseer Guds Hensigter med baade Fryd og Smerte , og jo lydigere hans Hjerte bsies efter Guds Villie , i Helliggjsrelse og Fred . Anvend derfor , aarvaagen og troende , de Midler , Gud har givet Dig til din Forcedling : foragt ikke heller den Almcrgtiges Tugtelse : den er ofte det kraftigste Middel — og vcer saa glad i Haabet , det faste Haab , som dagligen fuldbyrdes , at Al ting tjener dem tilgode , som elske Gud , at Intet vederfares dem « den Guds Villic og Styrelse — og hans Villie er Naade, hans Styrelse Viisdom . Han er den , som udsender Stormen , naar Luften behsver at renses ; han er og den , som efter Uveir lader Solen skinne . Tcenk da ei i din Prsvestund , at han forlader Dig ; se blot til , at Du ei forlader ham !

<h4>1956</h4>det betyder mindre end Intet , naar Spsrgsmaalet er om vorr sande , hsiere , evige Liv . Hvad stader os det , som vi her tabe , naar vi ikkun bevare Hjertets Fred og Reenheo ? Hvad gavner det og Mennesket , om han vinder den ganskeVerden , men tagerSkade paa sin <b>Sjel</b> ? Det Vigtigste er , for st at soge Guds Rige og hans Retfcerdigh cd , og troe , atdetOvrige tilfalder i det <b>Maal</b> , som er os nyttigt og fornodent i disse vore Provedage . Vor himmelske Fader veed vel , hvad vi have Behov . Han giver og han tåger , han opladcr og tillukker sin Haand , han beskikker Lys og Morke paa vor Bane , og fordeler paa mangehaande Maader vore Skjebner , for at vi aldrig maae indstumre i Sikkerhedens farlige Dvale , eller fa ' ste al vor Hu og vort Sind ved Stover , men betcenke , at vi her ikke have noget blivende Sted , at vi her ikkun ere Fremmede og Udlcendinge , som ved Ondt og Godt stulle opfostres og forcrdles til vort himmelske Borgerstab , beredes og fuldkommes til engang at indtroede i det rette Fcrdreneland . Han elsker os Me ligedan , og udstrcrkker sine Hoender den ganske Dag til Enhver , som hsrcr hans kaldende Rost . Dersom vi ei komme , men gaae hen til Jordens Erhverv og Sysler og Nydelser , som om disse vare vort hsieste , vort eneste Gode , saa er det os selv , som spcerre os Veien til hans Kjerlighed , saa er det Synden , som gjor Skilsmisse mellem os og vor Gud . Intet Andet kan stille os fra ham , ikke det Hoie , ei heller det Dybe , ikke det Ncervcrrende , ei heller det Tilkommende. Lader os derfor randsage vort Hjerte og see til , paa hvilken Vei vi ere . Om vi ere bortkomne fra det Liv , som er i Gud , om vi , fordybede i Verdens Omsorger eller henrevne af dens Lyster , have glemt vore Pligter , og ikke betcenkt , hvad der tjener til vor Fred : lader os , medens det er Tid , vende om til vor Gud , staae op og gaac til vor Fader . Han <b>moder</b> de tilbagevendende Bsrn med aabne Arme ; der erendnußum — men ikkun for dem , som arbeide , dem , som med uastadelig Aarvaagenhed og Bon , med from Benyttelse af de dyrebare Naademidler og aabent Sind for den gode Aands Virkninger,

<h4>2207</h4>Rige , hvor den hsieste Kjerlighed og Retfcrrdigbed er Tingenes Ophav , holder Verdens - Scepteret , leder Begivenhedernes Gang , uddeler Skjebnens Lodder , tildeler Enhver det <b>Maal</b> af Krcrfter , som for hans Bestemmelse er fornsdcnt , befordrer det Godes Fremgang , endog paa Veie , som for os synes übanede og morke , oph older , bcvogter , omfatter Alt og Alle , saaat Intet og Ingen forglemmes , forstydes , forkommcr , men hele det store Sammenhcrng og enhver Deel deraf opholdes af en huld og usynlig Haand , som aldrig trcettes , aldrig tåger feil , aldrig oph orer at vcere den skabende, beskyttende , som virker alt Godt , giver alle gode Gaver, leder Alt til en god Udgang . Naar denne opwftende Tanke besjeler vort inderste Vcrscn og gjenncmtrcenger os med hele sin levende Kraft baade i Fryd og Smerte , da have vi en Livsanskuelse , som , under alle udenfor os stedfindende Forandringer , vedligeholder sig lige trsstelig og recn . Storme kunne rase omkring os , men de skulle ei forstyrre Roligheden i vor <b>Sjels</b> Dyb ; de skulle , idet de fare frem , ikkun rense den Luft , i hvilken vi , desto friere , desto friskere aande vort evige Liv . Morke kan bedcrkke vor Vei , tykt , mangengang og uigjenncmtrcrngeligt ; men i vort stille , tillidsfulde Hjerte er det lyst , og bliver alt lysere , jo mere vi komme til Overbeviisning om , at dette Liv ikkun er en Opfostringstilstand , og at saaledes Alt , som vederfares her efter Guds Villie , er , i sandeste Betydning , til vort Bcdste . Milde og klare himmelske Stemmer lyde til os fra det hellige Ord , og vi hore dem , vi mindes dem , vi forlade os paa dem ; — Eders himmelske Fader veed , hvad I have behov ; kaster al Eders Sorg paa Herren ; som en Fader forbarmer sig over sine Born , saa forbarmer Han sig over dem , som ham frygte ; der falder ikke en Spurv til Jorden udcn hans Villie ; der vorer ingen Lillie paa Marken , som han ei nccrer , udvikler og klcrder i dens Pragt : ere I ikke meget mere end de ? Kan og en <b>Qvinde</b> forglemme sit Barn , saa vil jeg dog ikke glemme Dig , siger Herren . Om I , som ere onde , vide at give Eders Born gode Gaver , hvor meget mere

<h4>2335</h4>om Du ei kan blive « lykkeligere , end Du er ? O ! Du kan blive det , om Gud skulde sinde det nsdvendigt til dit eget Bedste , og om Du ei paa anden Maade kan reddes . Vcer tilfreds med , hvad Herren gjor , og loer i Alt at forstaae den <b>Lcrrdom</b> , han vil give Dig . Han tilskikker Intet forgjcrves, og han er allesteds ncrrvcerende . Kunde nogen Tid komme , da hans Kjerlighed ophsrte , eller hans Rigdomme udtsmtts , saa maatte og Du med forfcerdelig Uro gaae din Fremtid imsde ; men han bliver den , han er , en Fader og Forbarmer fra Evighed til Evighed . Kunde noget Sted findes i den vide Verden , hvorhen hans Blik ei trcengte og hans Arm ei naaede , saa maatte og Du stjcrlve for din Skjebne ; men nu — stjcrlv alene for Synden ! thi hvor skal Du henflye for Herrens Aasyn , dersom dit Hjerte fordommer Dig ? Men hvor , om der er Fred i din Samvittighet ) , hvor kan Du og verre faa übemcerket , at ikke han , den alvidende Forbarmer , fslger Dig , beskjcrmer Dig , kjender Dig . Hvorsomhelst Du seer et Grasstråa fremffyde aflorden , Hvorsomhelst Du hsrcr den mindste Fugt qviddre i Luften , der lever , der virker jo han , som endnn idag klcrder Lilierne paa Marken og fsder Fuglene under Himmelen ? Hvorsomhelft Du kan bede , verre taalmodig og gjsre eller idetmindste tcrnke det Gode , — der feler Du jo Virkningen af hans <b>Aand</b> , — - som tuller dine Hovedhaar , som troster Dig med sit Ord , — som raaber til Dig baade med Glcrdens og med Sorgens Rost : Kan og en <b>Qvinde</b> forglemme sitdiende Barn , saaat hun ikke forbarmer sig over sit Livs Son ? Ua om hun end forglemmer ham , saa vil jeg « dog ikke forglemme Dig . Se , jeg har tegnet Dig paa mine Hcender . En kjerlig <b>Moder</b> derimod giver ofte sit syge Barn et bittert , men gavnligt Lcrgemiddel , og det uforstandige Barn strceber derimod . Saa gjor og Du , Menneste! naar Gud sinder det nodvendigt at rcekkke Dig Bcdrsvelsens Kalk . Uden den stulde dog dit Hjerte ei blive helbredet , stulde din <b>Sjel</b> ei blive karst for Syndens Besmittelse til det Liv , som er i Gud . Betcenk dette , og ta g med stille Hengivenhed den lcrgende . Kalk fra en

<h4>2504</h4>et Ovelsesmiddel for vore Meste Krafter , et anbetroet Gods , som vi bsr forvalte med Omhu og Retffaffenhed, Reisepenge , som den Algode har forundt os paa Vandringen, til dette Livs Ophold og Tilfredsstillelse , en Forlening , som vi ei kunne beholde , men som vi bor formere etter forbedre, fsrend vi tilbagegive den , en uskyldig og nyttig <b>Sjelens</b> Sysselscettelse , medens den venter paa Sendebudet , som stal bringe den hjem . ' Paa denne Maade erholder vort Arbeide i Tiden en Retning til evige <b>Formaal</b> , et Preg afFuldendelse , Overensstemmelse og Verdighcd , som paatrykkes alene af Religionen , og som giver selv det forgjengelige Stof , vi bearbeide, de uforgjengelige Egenskaber af Godhed , Retferdighed og Sandhed , som ere Aandens Frugtcr og uden hvilke Intet har Betydning . Hvad der da msder paa vor Vei , ' det maa vere St-ort eller Lidet efter Verdens Maalestok , bliver os hverken for tungt eller for let , hverken for hsit eller for lavt ; thi , eftcrdi det <b>moder</b> paa vor Vei , saa er Guds Villie , at vi skulle modtage det , have Tilsyn med det , bruge det overensstemmende med hans hellige Bud , ' — det er en Herrens Gjerning , hvis Varetagelse eller Fuldbyrdelse har Fslger , har Forjettelser med sig , ei mindre for denne Verden end for den tilkommende , — og hvad tilkommer os Andet end at vcere faste , urokkelige, altid rige i Herrens Gjerning , efterdi vi vide , at vort Arbeide ikke er forgjeves i Herren ? Fra dette Synspunkt bsr vi betragte , i denne <b>Aand</b> bor vi indrette vore jordiske Forhold . Lader os med Glede omfatte Alt , som , erindrer os derom , vekker os dertil , opholder os deri ; lader os varetage vore verdslige Anliggender , men saaledes , at vi aldrig forglemme det Vigtigste . Fsrend vi begynde Dagens Besverligheder , maae vi hellige alle vore Foretagender med et hjerteligt Suk til ham , som alene kan velsigne . Naar vi mettes af hans gode Gaver og nyde hans rige Velgjerninger , maae vi tenke paa ham med levende Mrefrygt og stille Glede . Naar Sabbatsklokken bebuder, at Arbeidsugen er forbi , maae vi bortlegge de jordiske Tanker og Sysler og berede os til med Andagt at gaae gjennem hans Porte med Taksigelse , ind i hans

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011060704053'target=_>Essendrop, C.P.P., 1873, Prædikener til alle Aarets Søn- og Helligdage</a><h4>706</h4>Os Alle , som ere dpbte til Christus , har Herren kaldet til sin Vingaard ; os Alle har Herren engang sat ind i sit Rige og sat til Arbeidet . Hvorledes er det saa nu med os ? Kjende vi Alle dette Arbeides Krav og Pligt , dets Vyrde og dets Lyft ? Eller , er det gaaet saa med Nogen , at Dagen kom , da Sjoelen ikke fandt nogen Lyst , men kun Byrde , svcrr Byrde i Arbeidet , fandt , at Arbeidet var et Slid til Dpden , utaaleligt for Sind og for Kraft , og saa lagde Arbeidet ned ? Er det gaaet saa med Nogen af os , at han hverken nu staar i Arbeidet eller engang kan mindes , at han nogensinde har vceret i Arbeidet ? Da maa det jo vcere saa , at , da Arbeidets Tid kom , var han allerede den , som viste Arbeidet fra sig , var han maaste allerede bleven saa fremmed for Arbeidets <b>Aand</b> , at han hverken hsrts eller forstod Kaldet til Arbeide . Det kan jo ste , at det Barn , som engang saa godt forstod Moders <b>Maal</b> vaa Moders Tunge og selv byttede Ord med <b>Moder</b> i det kjcere , hellige <b>Maal</b> , vokser op til at glemme det ganske , saa Manden ikke kan tale det , ei forståa dets Lyd .

<h4>754</h4>Endelig , Medforlsste , — hvad vi her kun kunne antyde — derfor lyfter Herren Taalmodigheden saa hoit mellem Ordets Frugter , fordi Troeslivet er et Samfundsliv i Christi Menigheds Samfund , et virkeligt Samfundsliv , hvori Brodrene drives af een <b>Aand</b> mod eet <b>Maal</b> og derfor elske hverandre og i Kj«r-ligheden bcere hverandres og Samfundets Byrder , Byrder af de oprigtige Brsdres Svaghed , de falste Ondstab , de Udenforstaaendes Fiendstab mod Herren og hans Kirke , dens V ( rsen og dens Gjerning . Og Broderkjcerligheden er <b>Moder</b> til Kjcrrligheden til Alle . Troens Meend have Hjerter for ethvert Samfund , forn Gud har knyttet , for det Arbeide og det <b>Maal</b> , forn Gud har fat for dette Samfund . Og hvert Samfund har sin ' Byrde og derfor sin Smerte , og overalt hvor der er Smerte , der er Troens <b>Aand</b> med sin Kjcerlighed og tager Smerten og bcerer den i Taalmodighed , og Herrens Velsignelse er i den . Saa er Svaret vaa vort SvMgsmaal givet i Korthed , og vide vi end godt , at der hefter megen Skrsbelighed ved Befvarelsen, vide vi dog , at der er Sandhed , Guds Sandhed i Svaret , og saa bede vi Gud selv velsigne for hver af os Have vi i vore Hjerter erfaret denne Taalmodighed som Ordets venlige Frugt ? Herren selv givs os det rette Svar og Svarets rette Frugt !

<h4>861</h4>Kvinden kom til ham , og hendes Tro vokste sig stor . Trods Alt , hvad hendes Erfaring sagde hende om , hvorledes Ipderne pleiede at behandle Hedningernes 2 Et , ja trods Herrens tilsyneladende Ukjcerlighed , trods hans Afstag , bley hun dog bestandig i Bemnen . Hun opgav ikke Troen paa , at han baade kunde og vilde hjcelpe hende , hvor saare ringe og uvcerdig hun end vidste sig selv at vcere . Almindeligvis gaar det jo ganske anderledes til . Saa Mange ere der , som slet ingen Tro , ingen Tillid have til Herren , som langt hellere forlade sig paa sin egen eller Andres Styrke og <b>Visdom</b> eller paa den blinde Skjcebne , hvorom de daarlig tale , end paa den levende Gud . Andre ere der , fvm nok paa sin <b>Vis</b> fortrsste sig til Herren , saalcenge de gode Dage vare , men , naar de onde Dage komme , er det saa saare snart ude med deres Tillid og Haab . Det er dem strax , som om Himlen er aldeles lukket , som om der ingen Herre er , hvis levende Yre opfanger deres Raab , hvis mcegtige Arm er udstrakt over dem . Ja , vi vide det vel Alle , at det er let at tale om Herrens Barmhjertigheo og Kjcerlighed , naar der ikke er Noget , som trykker vort Hjerte , Intet , som volder os Sorg og Bekymring ; men det er ikke let , naar det just gjcelder at tro og ikke se , at holde fast med Haab mod Haab , at bie paa Herren , naar han ne < ler med at komme ; og det er ikke let , fordi vi ikke ret have annammet Herrens Ord og derfor ikke ret have stuet ind i hans Kjcerligheds og hans Magts Dyb . Derfor se vi , naar Vandbplgerne true , med Peter hen paa os selv og paa Bplgerne , ei paa ham , som kan binde Belgen og kue Stormen . la , hvem er der vel mellem os , som ikke maa bekjende dette om sig selv ? Derfor taler Dagens Evangelium straffende Ord til os Alle .

<h4>2249</h4>foragte eller forsømme , hvad der ogsaa lcegges os paa Sinde i Herrens Ord i Dagens Evangelium : ~ Og han bpd d em , at de ikke skulde sige Nogen det . " Men mcv al Fastholden heraf Vliver det jo dog mere end paafaldende , hvorledes de Mange , Mange have saa Meget at sige om alt Andet end om Synd og Naade , om Guds Kjoerlighed , om Jesu Kors , om Aandens Veie , og , naar Talen fpres derhen , hvor ofte Ordet da bliver saa mat og fattigt , og selv hvor en naturlig Veltalenhed fsder Ordet og paa sin <b>Vis</b> lsfter det hsit og rigt , hvor ofte Het dog fornemmes , at det kun er en Veltalenhedens Kunst eller en Indbildningskraftens Flugt , og Ordet kunde kun beruse og henrive for et Beblik ; der var dog ingen Kraft til blivende Virkning deri . Du <b>moder</b> en Anden maaske med liden verdslig <b>Dannelse</b> , med lidet Tankeforraad og snever Synskreds , men taler du med ham om de Ting , som hsre til Guds Rige , har han dog baade Tanter og Ord , og Tanken kan vcere enfoldig , og Ordet saare kunftlpst , og dog kjendes der saa ofte en gribende Magt i dette kunWse Ord . Det var Menneskets <b>Aand</b> , der var i hines Tale , Guds <b>Aand</b> , som rprte sig i dette Ord . Men Guds <b>Aand</b> er i deres Hjerter , som i Christi Tro ere blevne forligte med Gud og gjenfpdte til Guds Boms Arv , og i deres alene . Han , som har sonet vor Synd og dermed nedbrudt Adskillelsens Mellemvceg mellem os og Gud , han , som med sin Frelses Kraft lsser vor Trceldom under Synden og giver os sin og Faderens hellige <b>Aand</b> , ham er det ene , som er vore Sjceles Lcege . Naar Gud i Christus kommer og frelser os , aabnes de Dpves Uren , og den Stummes Tunge synger med Fryd .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111103001'target=_>, 1856, Christiania-Postens Feuilleton</a><h4>1975</h4>broen . Da enhver Feber voxer til den nåar sin Crisis , san brandte han altid hedere og hedere i sin Utaalmodighed . Han lod snart Hestene galloppere , og de muntre Heste gjorde det med Glade , de vare stolte af at <b>vise</b> deres Kraft , men dette gjorde deres Huusbond vildere og vildere . Da nu Hestene vilde <b>vise</b> , at de ikke havde glemt deres Tugt , og holdt stille hvor de vare vante til at holde og gik i Skridt op og ned over Bakke , saa tabte Bent reent Teilen over sin egen Forstand og jagede i Fiirsprang op over og ned over , saa Hestene dampede som Dampkjedler . Djcevelen sat » bag Vognen og passede sit Snit . Hestene have Lighed med Mennesker . Mble Heste , som ikke ere nedvcerdige af en Hesteplager, yde deres Herre i Nodens Time den utroligste Bistand , de udrette hvad der synes umuligt , de offre fig , de gaae under , de synke om ved <b>Maalet</b> , men deres Herre er reddet . Imod umenneskelig og slet Behandling derimod reiser sig ethvert Håar paa deres Legeme . Havde Bents Heste kunnet forstaae , at deres Herre vilde ester sin Elskede , saa havde de lebet i Doden for ham , men det kunde de stakkels Dyr ikke fatte og begribe , og saa havde Djavelen et let Arbeide . Hestene stode stille , nåar Bent saa uvorent pidstede paa dem , de gik tilbage nåar han rev i Tommerne , og jo meer rasende Vent blev desto trodsigere bleve Hestene . Da blev Bent rammet af et Hovslag og faldt under Vognen, og da greb Djavelen ham og foer afsted med ham . Bent saa aldrig sin <b>Pige</b> igjen . Der var stor Sorg paa Jorden over Bent , og hans <b>Pige</b> var fardig at grade sine Oine ud . Derfor var der endnu Forbarmelse for Bent . Det blev ham lovet , at dersom han engang kunde omvende en saadan Hesteplager , en saadan morderlig Vognmand , at han ikke meer pinte sine Dyr , at han ikke overgav sig selv til Djavelen og aldrig bar sig varre ad end det Dyr , han mishandlede , saa stulde Bent komme los fra sin Ildvogn og igje » forenes med sin Elsi . De hellige Natter ere hans , da bliver han lesladt og da kan han forsege

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009072003037'target=_>Luther, Martin, 1864, Luthers Huuspostil</a><h4>6580</h4>Her horer du , hvorledes hun fryder sig , og hvor frimodig og glad hun er . Thi hun siger , at der er Noget , som hendes <b>Sjel</b> ophoier , det er , som hnn maa love og prise , og hvorover hun maa fryde sig saaledes , at der i hendes Hjerte ikke er en Tanke , uden at den jo er opfyldt af Glede og Fryd . Hvad er det nn , som hun saaledes ophoier ? Sig selv ? Nei , siger hun , jeg selv er Intet , og vil heller ikke selv gjere mig til Noget . Og nåar derfor du , kjere Elisabeth , priser mig salig , og kalder mig den Velsignede blandt <b>Qvinderne</b> , saa priser og ophoier jeg herfor alene Gud . Thi hans er det Altsammen, og ikke mit . Her legger huu altsaa eu meget skjen Betydning i det Ord  at opheie " ; thi hvo der ophoier en Anden , han fornedrer og ydmyger sig selv ; og da hnn nu siger :  Min <b>Sjel</b> ophoier Herren " , saa bekjender hun dermed , at hun underkaster sig Herren , og hverken ophoier eller hovmoder sig selv . Hermed rammer hnn ret os arme Staadere , den Ene saavelsom den Anden , St . Paulus saavelsom mig og dig , fra vor svage Side , og tager os alvorligt i Skole . Thi hvad gjore vi ? Det almindelige Mnndheld blandt vore Borgere og Vender er bekjendt nok : Jeg har dog endnu et Par Skilling at tere paa . * ) Ja saaledes roser man sig af Penge og Gods , Gylden og Dalere . Men hvorfor hovmoder dn dig dog heraf , du arme Staader ? Burde du ikke sige : Det er fremmed Gods ; Gud har givet det , han kan ogsaa tåge det igjen ; hvorfor stulde jeg da hovmode mig deraf ? Iligemaade gaaer mangen heilerd Kjetter og bryster sig af sin <b>Lerdom</b> og sin Skarpsindighed. Men siig mig : Hvem tilherer

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092303042'target=_>Rosenius, C.O., 1868, Hemmeligheder i Lov og Evangelium</a><h4>204</h4>Men lad os derfor ogfaa endnu noget bedre betragte de Gaver og Evner , Apostelen her har nevnt , og om hvilke han udtrykkelig har sagt , at nogle Lemmer i Legcmet havde dem , men andre ikke . Han ' sagde : „ En gives Visdomstale formedelst Aanden , en Anden Kundslabstale ved famme <b>Aand</b> . " <b>Visdom</b> og Kundftab eller . . Forstand " vare saaledes tvende forskjellige Gaver. <b>Visdom</b> var den Gave at skue md i Guds Verk . ' Guds Veie , Domme og Hemmeligheder med et dybt og klart Blik . Kundstab derimod var den Gave at anvende Ordets Sandheder paa Mennester paa en heldbringende Maade og i hvert serskilt Tilfelde sinde , hvad der mest gavnede . Der hsrer f . Ex . <b>Visdom</b> til at knnne forståa og forklare Johannes Aabenbaring eller andre dybe , hemmelighedsfulde Texter . Men nåar Apostelen Petrus siger , at en Mand stal „ boc hos sin <b>Hustru</b> med Foruuft , " saa bruger han der just det Ord , som her er oversat med Kundsiab", og vil sige saameget som , at Manden bnrde indsee og betenke det „ <b>kvindelige</b> Kars " Svaghed og derfor anvende Maadchold og Billighet » i sine Fordringer . Ligeledes nåar Paulus vidste at stikke stg efter alle Omstendigheder , vere „ dem under Loven , forn den , der var under Loven , " vere  de Skrsbelige som en Skrobelig , " vere „ Alt for Alle , " faa horte alt dett . ttl den Gave , forn her kaldes Kundstab eller Forstand . Kundflabstale er faaledes at kunne gjore ret Anvendelse af Guds Ord paa forskjellige Mennesker , paa en mild eller en skarp , en trostcnde eller truende Maade , efterfom Enhver behover det . Hvorimod Visdomstale var den Gave med et dvbt og klart Blik i Ordets Hemmeligheder at gjore dybfindige Uolegninger deraf til Menneskers Undervisning . Nu bsr man dog inc forståa dette saaledes, at Nogen havde den ene Gave i stort <b>Maal</b> og aldeles intet as den anden ; nei , disse Gaver kunne vel vere forenede , som hos Paulus selv og Flere ; men Apostelen vilde sige , at Aanden gav den Ene et stort <b>Maal</b> as <b>Visdom</b> , saa at denne fremtraaote som hans serlige Gave , medens han dog kun havde et meget ringe <b>Maal</b> af den Gave , fom kaldes Forstand . Derimot » knnde en Anden have denne sidste Gave i rigt <b>Maal</b> og dog kun have meget lidet af den fsrste . — Videre siger Apostelen : „ En Anden Tro ved den samme <b>Aand</b> , en Anden Gaver til at heldbrede ved den samme <b>Aand</b> . " Med Ordet Tro menes her udentvivl den undergjorende , saasom nåar Aposteten i neste Kap . v. 2. siger : „ al Tro , saa at jeg kunde stytte Bjerge " jevns . ( Math . 17 , 19. 20. ) men det kan tillige sigte til , hvad man ogsaa seer af Historien, at Gud stundom har givet nogle forresten ringe og «bemerkede Kristne en besynderlig Heltetro til at vove store Ting

<h4>289</h4>dsd , treg , uacmdelig , Arbeidet frugteslsst , ligesom det altid maa see ud paa den Ager , hvor man kun har brudt og ploiet Jorden , men aldrig endnu udkastet Seden , samt hvor Regn og Sol endnu ei ere komne til . Men nåar han vender Bladet om og forkynder lefus Kriftus og ham korsfestet , prediker Syndernes Forladelfe for dem , forn ligge ulykkelige under deres Synder og endnu ei have overvundct dem , da begynder det at rores , gronncs, knoppes , da skulde altid nogle forhen haarde Hjerter smelte , blive bsiclige . ydmygc , glade , elskende , villige og skikkede til god Gjerning . Ja det er denne Artikrl , som <b>mor</b> os salige , . par jeg mine Synders Forladelse , saa er jeg allerede lykkelig , allerede ct Guds Barm — og hvad kan jcg merc blivc eller onstc mig i dctte Liv ! Har jeg ' ilke Syndernes ' Forladelse , hvad garmer mig da alt Andet ? * ) — O , hvvr nlykkeligt er ikke det Menneste , som ci har sine Synders Forladelse ! han er altid under Guds Vrede og Forbandelse — forbandet ved sin Udgang og Indgcmg , forbandet i Hufct og paa Marken , forbandet i Gleden og i Sorgen , forbandet i Arbeidet og i Hvilen , ja forbandet i Livet , forbandet i Dsden og — i Evigheden . Men rettelig at lere ug umsattc denne Artikel , med virkelig lcvcudc Tru at omfatte og dagligcn fastholde sine Synders Forladelse, det er ogsaa de Kristnes allcrsverestc Kunst , hvori dc aldrig blive udlertc alle deres Livsdagc , ihvurvcl de daglig ove stg den med Lesning , Horen og Tale om Kristo og den store Naade dc have i ham . Dc forblive dog altid spede Disciplc i d ^ n Knnst for egen Del at tro og tilegne fig denne . Dette kummer deraf , at dc lige saa daglig og uophorlig scc og fole saamcgcn Synd og Skrsbcltghcd hos sig selv og saamegcn Mangel i Kristendommen , medens derimod Naaden og den evige Retferdighed i Kristv ikke synes ellcr fslcs , det kommer deraf , at dcres Hjerter — som forhen vare saa trygge , Mre og sterke — nu ere sondcrknusic , svage ug bevende ved Herrens Frygts <b>Aand</b> , som boer i dem , endelig kommen det ogsaa deraf , at deres Tro , som nu er saud og saliggjorcudc, just derfor uu er bleven <b>Formaal</b> for Satans uophorligc Anfegtelser og gloende Stud . Af alt dette kummer det . at ingcu sand Kristen er saa oplyst i Evangelium eller er kommen saalcmgt i Troen , at han ikke stnndom lidcr af Frygt ug Bekymring for fin <b>Sjels</b> Tilstand , ja at dc sturstc Helgene , ' som have talet ^ mcd Euglctnngcr om Kristi Blods evige Verd og Fortjeneste , dug ofte sclv have veret svage , anfegtcde og bckymrcde og kun mcd Msic have klmuct tro , hvorpaa David , Paulus og Luther m . fl. cre talende Excmplcr . Men dette kunne de fleste Mennesker itkc begribe ; thi de

<h4>612</h4>om han blot kunde bestrive , hvad der er ham selv saa kjert , da stulde ogsaa Andre sorstact det , see hvad han seer , og tomme til Sandheden ; da taler han itke med kunstige Ord efter Menneskers <b>Visdom</b> , men med den elskelige Enfoldigheds Sprog tan han nu paa naturligste Maade og af egen Erfctrmg beskrive f . Ex . Syndefsvnen, Siklerhedens Stotter og vrange Meninger , Guds Ord og Domme , Synden og dens Folger eller den store , uforskyldte Naade i Kristo , samt hvor stor og inderlig Glede det er at vere bekjendt med Frelseren osv. sandes Luthers Ord , at . . Evangelium soder ittc saadanne Born , som ligge stille i Vuggen , men raste Anglinge , som ogsaa ere dygtigc tit at omvende Andre ; " thi „ de stulle alle vere fodte af Gud baade Smaa og Store , " og af denne Aandens Lerdoru taler da en ellers Ulerd langt bedre om de guddommelige Ting end dc mest Hotlerde , som endnu selv ere uomuendte og uerfarue . Meu Mctctden , Gaverne vg Kaldclserne ere forskjellige ; thi ikke alle Lemmer i Legemet have samme Embcde ; Oiet seer , men kan ikke hore ; Fodderne gaa , men kunde itke tale ; Tungen taler , men gaacr ikke osv. Den Eve har den Gave at tale med egne Ord til Sjetes Opdyggelse af Gnds Ord , venlig og viselig at rctade , advare , vekke , op ! yse og beskrive de aandelige Ting ; den Anden har „ et tungt Mele " ligesom Moses , men lan vere virksom til at lese i Bog , paapege vigtige og vekkende Stykker ; en Tredie taler meget tydeligt med sin alvorlige , gudelige Vandel, hdormed ogsaa alle Troende med mere eller mindre Held umage sig for at predike , hvurvcd de beftraffe og — forn ogsaa Verden erkjender — virkelig . . domme , " ogsaa da , nåar dc ei sige et eneste Ord med Munden ; den Fjerde er et uophorligt bedende Hjerte i Kristi aandelige Legeme , som paa denne tause Maade med Forbsnner gjor Guds Gjerning ; den Femte er motter og virksom ved gode kristelige Indretninger og Stiftelfer . Dette fynes i vor Tid at svare til Apostelens Ord : „ En gives Visdomstctle formedelst Aanden , en Anden Kundstabstale ved samme Aano — en Anden Tro ved den samme <b>Aand</b> , en Anden Gaver til at heldbrede — en Anden Kraft til Undergjerninger , en Anden profetisk Gave " osv. Herved maa dog merkes , at dette bor ikte forstaaes saaledes . som om Enhver kun kunde virke paa een Maade og ingen Del have i den Andens Gave ; men her omtales kun at Enhver har sin ndmerkende Gave ; f . Ex . den , som savner Gave til i Almindelighed at lere og undervise med Ord . og har en treg Tunge , han kan dog tale noget , kan dog ligesom de bethlehemitiste Hyrder . . kundgjsre " paa Fctctrehyrders simple Sprog , hvad de have erfaret om Barnet , eller forn <b>Kvinderne</b> forkynde , hvad de have feet ved Jesu Grav , lobe med en Hilsen fra den opstandne Frelser og sige forgende Sjele en glad Tidende , ihvorvel det steer paa et <b>kvindeligt</b> og enfoldigt Sprog . Saa maa de jo og alle predike med Levnet , og Alle maa jo

<h4>649</h4>Aand stulde bo i mig , det var da den groveste Dactrstab ! Nei ikke Guds Acmd . men Djevelen . See , nu er sandelig vort Liv ntcd Kristo ret dybt stjult . Endelig kommer endnu hertil , at Gud lader allchacmde übehagelige Erfaringer . Ulykker og Lidclsir mode os . ja forn en brusende Flod overskylle os , saa at alle Magter , Naturen , Mennesker og Acmdcr paa engang forene sig mod en from Job , idet Rovere , Stormveir og Torden frcttage ham alt , hvad han eier , endog Bornene ; Djevelen plager hans Legeme , hans <b>Hustru</b> befpotter hans Tro , hans arme Trostere velte nye Stene paa hans Byrde , og hans eget Hjerte rctfer i Gudsbespottelse , saa at han forbander sin Fodselsdctg . O , hvor er nu den hott benactdede Mand , hvis Lige ikke fandtes i Landet , hvor er nu det ferlige Guds Venstctb , han stulde eie ? Saadant maa vel hedde at skjule vor Herlighed paa det ctllerdybeste . Men af alt dette Onde , forn bedekker og skjuler vor Herlighed, vort Liv med Kristo , er intet at ligne ved Synden . Idre Lidelser ere gyldne Lidelser derimod . Man kan temmelig snart lere , at disse ere et saderligt Ris , at „ den , Herren elsker , dcn revser han ; " men Synden , Djevelens Rasen i Kjodet , vedholdende Syndelyster og den derpaa folgende dsde Fslclse og Forestilling om Guds retferdige Ovcrgivelfe , om Aandens Bortvigen m . m . fee disfe ere de rette Dodsstod , som gaa os igjennem Mctrv og Ben og i Grund formorke vort Nacrdcliv . Da hjelper sedvanlig intet andet end at opgive al Tanke paa sit eget Nactdeliv og blot see paa den evig uforanderlige Gud felv , om ikte han kan frelse og endnu hjelpe Sagen . Men nåar det atter lysner , kan man vel faa at fee , at midt i det tykke Morke stjulte fig ikke blot et uforandret Faderhjerte i Gnd og vor egen nformindstede Retferdighed i Kristo , men ogsaa et sandt , levende og lempende Nactdeliv i vort Hjerte . At forståa sig vel paa , hvvrledes Gud forer sine Hellige underlig og paa denne Maade skjuler Livet under Doden , Retferdigheden under Synden , Himlen under Helvede — det er  <b>Visdom</b> for de Fuldkomne", det er den hoie Guds <b>Visdom</b> , forn fremfor alt andet er os nodvendig , om vi stulle holde ud med denne Herre . Herhid horer , hvad Luther taler i Ep . Post . paa Joh . d . Dobcrs Dag , da han iblandt andet siger : „ I have oste bort , mine Venner! at Gud lader Stitkelfen og Folelsen af Syno , Dod og Djevel blive hos os , saa at Synden gnaver mig , cmgriber mig i Samvittigheden og vil drive mig til Fortvivlelse ; Guds Dom forstrekker mig ogsaa , ligeledes Doden , som vil opsluge mig , og Djevelen treder mig paa Halsen og vil nedtrykke mig . Denn ? udvortes Skikkelse lader Gud blive tilbage og borttager den ikke ; dette Udfeende maa blive , saa at vi intet andet fole , end at vi ere Syndere og ligge under Doden og Djevelen . Og under denne Skikkelse bliver Liv , Uskyldighet » og Herrevelde over alt dette Onde dug tilbage , som Herren sclv siger i Math . 18 : Heldedes Porte stulle itle faa Overhaand over min Kirke . 49 ° "

<h4>774</h4>Men her turde Nogen cmmcerke : Kjcrrligheden lader stg jo aldrig fremtvinge . Det , jeg elster , det elster jeg , ont det end forbydes; det jeg tkke elsker , kan jeg ikke tvinge mig til at elste . om end Gud selv staaer for mig med Svcrrdct og befaler mig det . Hvorledes kan da Apostelen formane til Kjcerlighed ? Svar ; Vi tale jo her kun om Kristne ! Lad os ihukomme , hvad det betyder : „ Om I ere opftandne med Kristns " — „ om Kristus er Eders Liv ! " Saaledes begyndte vort Kapitel . En Kristen , som staaer i Pagt og Forening med den almcrgtige Herre , har Veie og Midler , som ingen Anden har , til at faa , bvad han behover , som Kristus siger : Alle Ting ere mulige for den , som troer . Han kan faa Kjcerlighed efter Behov , ogsaa der , hvor det for den blotte Natur er umuligt at elske . Hcerligheden beroer ikke paa <b>Formaalet</b> , men paa Hjertet ; vi fee , at ogsaa dc foragtcligste Formactl blive clstcdc . Behover da en Kristen mere Kjcerlighed , saa — taler han derom med sin Gud ! Dette er det fsrste Middel ; thi Kjcrrligheden til Magen er ogsaa en Gave af Gud , forn man vel maa bede om . Det andet Middel er at betragte Guds Ord om Sagen eller betragte fin Mage efter Ordet . ' Det tredie eller rettcrc ' det forste og sidste er blot at oplives i Forholdet til Gud , i Troen ug K ' j « erlighedcn , da folger ogsaa Kjcerlighed til Magen . O , det er en undcrfuld Omstendighet», liflig ug dyrebar at kjende , kun bekjendt for de Troende, og hvorom kristelige Mager ofte med Forundring samtale , nemlig at deres Mgtekjcerliqhco staaer i saadant Sammenhceng med Forholdet til Ond , at just som dette sidste er , saaler og den fsrste . Blive dc kolde og dode i det <b>Aandcligc</b> , da bliver det ogsaa koldt og dodt i Mgtekjcerlighcdcn , ville dc da have en mere oplivet Kjcerlighed indbyrdes , saa ssge dc blot en oplivct Tro og Kjcerlighed til Gud . Her secr Du , hvad det bctydcr , at den sande Mgtckjaerlighcd er en hellig Kjcerlighed og har sin Grnnd i Gud . Men oa alt Helligt cmfcegtcs af Djcevclcn , og han aldrig kan ltdc , at et faadcmt Paradis , som tvcndc kristcligc Magcrs indbyrdcs Kjcrrltghcd er , endnu sindes paa Jorden , saa forssmmcr han ikke at gisre Anfalt » paa denne ; da bliver det nodvendigt , at de bestandig vel betragte hverandre efter Guds Ord og Indstiftclfe . Herom siger Luther i en Forklaring over Math . 5 , 27 — 30. fslgende Ord :  Kjso og Blod er forst fnldt af Nysgjerrighet » , faa at det kjcdcs ved , hvad det har , og gabcr ester noget Andet ; da puster Djcevclcn dertil , saa at man ttkc secr cmdct hos sin Mage , end hvad der er strsbcligt , og taber af Sigte , hvad der er godt og bcrsmmeligt . Dcraf kommer fnart , at enhver anden <b>Kvinde</b> er stjonncre og bcdre i Ens Oinc end hans egen . Da er det den rette Knust ' og det stcerkcstc Modvcrrn derimod , om enhver lcerte ret at see paa sin Mage efter Gnds Ord ; da stuldc han vel clste og agte sin Mage som en gnddommclig Sljcrnt uZ Klenodie , og nåar han saa en anden <b>Kvindc</b> — om hnn end varstjonncrc end

<h4>784</h4>til Gavn . " Men den uomvendte Elster har ogsaa et Haab , som — desverre ! — ufte er gaaet i Opfyldelfe , ug de Tvendes Kjod har overvundet den Enes <b>Aand</b> . Om da en Kristen bliver spurgt om Raad ctf en Broder eller Ssster mcd Henfyn til VEgtcstab med en Uomvcudt , hvad stal han da svare ? — Svar f . Ex . saaledes : Har Du noget Valg , kan Dn gjore hvilket somhelst ? Er Du ikke allerede bunden af Kjerligheden ? Kan Du blive fri , faa valg heller det ( 1 Kor . 7 , 21 ) . Bed Gud om , at Du flipper fri ! Og „ hvor veed Du , <b>Kvinde</b> , om Dn kan frelfe Manden ? eller hvor veed Du Mand , om Du kan frelfe Hustruen ? ( v. 16 ) - Nei , fly , fly . fly , om Du lan ! Men — det bliver dug — ctntctg , at det dog bliver til Giftcrntctal — hvad stal man da sige ? Ja hvad stal man da sige , om ikke saaledes : „ Gud styre nu ! Ber nu Dit Kors med Tactlmodighed og bed Gud , at hau frelser Din <b>Sjel</b> ! " — Hvorfor en faadan Slutning paa Spsrgsntctctlet ? Fordi intet Andet hjelper , fordi at forn oftest ingen Kraftord , Advarsler , Bonner eller Forbsnner hjelpe . Og om den Troende endnu vil folge sin Frelser , hvem kan da negte ham , at han endnn har hans Naade ? Men hermed sammenhenger et andet Spsrgsmaal ; mangen en ung Kristen siger : „ Om der end ikke er Spsrgsmaal om ' en Vantro , er jeg dog uvis , om Gud har bestemt den for mig , forn jeg elster . Hvorledes stal jeg da vide Guds Villie ? " Her have vi et godt Raad at give . Men fsrst et Ord om hette , at Du virkelig fporger efter Guds Villie . O at alle unge Kristne vilde dybt legge paa Hjerte , at dette er det eneste vigtige i denue Sag , nemlig at Guds Villie steer ! At en Person er kristelig og dernest prydet med alle gode Egenskaber , er aldeles tkke noi ; Du behover endnu at vtde , om dette ogsaa just er den Person , forn Gud har bestemt for Dig ; thi for det Forste kan man jo ikke engang fee til Morgendagen , hvad der i Fremtiden can stee med den Person , Sygdom , Dod , Frafald m . nt . Der er utallige Tildragelser , som kunne gjore Din Lykke tilintet . Men om Du blot vced , at deune er Dig givet af Gud , faa maa dn komme Sorg eller Glede , det er dog idcl Lykte . Derfor sagde engang en provet Kristen : „ Dersom Gud bdo mig frit at velge ! Mage , ont han opstillede i en Nekte alle <b>Formaal</b> for mit Valg ! og fagde : Tag nn , hvilken Du vil . da vilde jeg svare : Nei , Du gode Gud ! velg Du til mig : jeg seer ikte til imorgen , hvad der tan stee ; gtv mig den , Du har bestemt til mig ! " — Desuden, hvor vigtigt for alle kommende Anfegtelfer at kunne sm med Vished : Denne veed jeg . at Gnd selv har givet mig ! H sporges : Hvorlcdcs kan man komme til en saadan Vishcd ? Il hor ! er Du bekjendt mcd Frelseren , saa fremleg Din Sag foi ham i alvorlig Bon om , at dersom den Person , Du tenker pm er den rette , som han har bestemt for Dig , at han da vil giv Sagen Fremgang , men at han i modsat Fald vilde i sin Nacck

<h4>815</h4>kjerlighedsfuldt tale med dent derom , livligt fortelle dem de vigtigste af Bibelens Tildragelser f . Ex . hvorledes Gud ved Sinai udgctv sin Lov . hvorledes Frelseren fsdtes i Bethlehem , hvorledes han led og dode for vore Synder , videre om Englene , om Himmerig og Helvede , om den yderste Dom m . m . Naar disse og deslige Hovedpunkter af Kristi <b>Lerdom</b> paa en alvorlig , from og livlig fortellende Maade indplcuttcs i det aabne Bctrncfind , faa er dette en god Udsed for en fremtidig Host , om den end ikte saa snart stulde <b>vise</b> silt Spire . Men hvad der da videre var nsdvendigt, er at Foreldrene ogsaa med hele deres Vandel indprente det samme , som de mcd Ord have lert deres Born , at de itkc ved et stjodeslost og syndigt Levnet nedrive , hvad de have op bygget , og giuc deres Oorn Anledning til at tente , at de ikte selv tro , at de ti Bud ere Guds Ord , at Gud er allesteds nerverende o . dcsl . O , hvilken Ildpine har det ikke foraarsctget i mangen Faders og Moders Hjerte , at de ved Erfaringen af deres Boms Foragt for Herrens Ord have maattet nodes ' til at tenke : Hvem vecd , om itkc just jeg med mit ikjsdeslose Levnet har givet dem Anledning til saaledes at ringcagte mine Lcerdomme ! De have paa mig villet see Gudfrygtighedens Kraft og Betydning , men hvorledes har jeg bevist denne ? Ak vee , mactste jeg er Actrsag til deres evige Fortctbelse ! See mcd saadanne Ildfttle kan Djevelen vel altid bestyde vorc Hjerter , men iser om vi ved nogen serdeles Stjsdesloshed har givet denne „ Antlager Rum . " Men fornden de gode Lerdumme og det kristelige Exempel have Foreldre endnu etovermaade vigtigt Middel at anvende for deres Boms timelige og evige Vel , nemlig Forbonnen . Beder man uden Ophsr til vort Livs Gud for tusse dyrcbctre Planter ? Alt vort Arbeide er forgjeves, om itkc Herren felv giver sin Naade og Velsignelse . De bedste Lerdomme blive et dodt , uvirksomt Bogstav , de kjere Bsrn blive fordervede lige imellem vore Hender , om ikke Guds <b>Aand</b> indtager deres Hjerter og der virker , hvad Lerdommene indeholde . Desuden kan ingen menneskelig Omhu , tutet menneskeligt Oie , ikte vor bedste Omsorg eller Aarvaagcnhed allcvegne folge Menneskebarnet og bevare oet fra det Onde ; det maa engang slippes ud af Foreldrehjemmet , og fendes forn Lam iblandt Ulve ; hvem stal dct vere deres Fader og Hyrde om ikke han alene , forn er en . . Fader over alt , hvad Fader hedder ! " Tillige er det af stor Velsignelse , at vi ci alcne bede for dem , men ogsaa med dem eller lade dent felv ved visfe Tilfelde lytte til den hnlde Forbon . Schriver beretter , at det git som et Sverd gjennem hans uuge <b>Sjel</b> , da hau horte sin <b>Moder</b> i Nattens tansc Timer at ligge og bede for sine Born , og hnn omsider serskilt nevnede „ lille Christian . " Man leser ogsaa i mange store Mends Lcvnctsbestrivclser, hvorlcdcs dc tilskrive en from Mudcrs tidlige Lerdomme og Forbonner de fsrste Virkninger af Guds <b>Aand</b> paa deres Ajcrter . Ja omsider bliver Forbonnen det sidste og cncste som

<h4>1026</h4>Jorden ? « Liden og ond er min Elcndighcds Tid " — det var hans egen korte Bestrivelfe derover . Vi ville tale om en endnu mere udmerket Guds Mand , om bvcm Herren felv bevidncde : « Hans Lige er ikke i Landet ; retferdig front og skyer det Onde . " Job var derhos en meget rig og megtig Mand , havde mange Bsrn og sture Eiendomme ; men Satan sik Lov at rsre ved alt det , han ciede , og da borttoges det altsammen paa een Dag . Den . som vil see , hvorlcdcs et troende Hjerte bestormes i Anfegtclfcns Tid , han lefe Jobs 1 Kap . hvorledes det ene Sorgebud kommer efter det andet , saa at der staacr tre Gange : « Og medens denne endnn talede , kom den Anden , som sagde " etc. « og niedcns han endnu talcde , kont en Tredie " osv. og det cnc Ulyttesbndskab stotre end det andet ; den Forste bcrcttcde , hvorledes Araberne vare faldne ind i Landet og havde drebt Jobs Hyrder , den Anden , hvorledes Gnds Ild havde opbrendt hans Faar og Drengene , som vogtcdc dem , den Tredie , hvorledes Kaldeerne havde rovet alle hans Kameler og drebt Hyrderne , den Fjerde , hvorledes Huset var nedstyrtet og havde drebt alle hans Born . Da Job bar alt dette mcd Tactlmodighcd, fik Satan derefter Lov at plage hans Legeme med onde Bylder og Saar fra Fodbladet indtil Hovedisscn ; hans <b>Hustru</b> antaster hans Tro og Taalmodighed mcd bittre Hentydninger imod Guds Trofasthet ) ; hans seluretferdigc Venner arbeide paa at legge Sten paa Byrden med idelige Antydninger om , at alt , hvad han led , var Straf af Guds Retferdighed for hans Synder . Og endelig stal dct dyrebareste , han havde , hans Taalmodighed og Fromhed svigte ham — dan forbander sin Fodselsdag , sit Livs Bevarelse og dermed al oen Guds Godhed og Trofasthed , som havde givet ham dette Liv . For en faa from Mand maatte dette sidste vistnok vere det verste af alt , hvad han led . Saaledes lader Gud martre , sonderhugge og salte Kjodet hos sine kjereste Venner . Og saa forklarer den underlige Herre tilsidst , at samme Job var ham saa tekkelig og behagelig , at dersom de tre Jobs Venner , som havde plaget hans <b>Sjel</b> med deres hoie , men falstc <b>Visdom</b> , stulde vinde Guds Naade , faa stulde Job bede for dem — - « Lader min Tjener Job bede for Eder ; thi jeg vil ansee hans Person , at jeg ilte stal gjore imod Eder ester Eders Dactrlighed ; thi I have itte talet ret ont mig som min Tjener Job " ( Kap . 42 , 8 ) . Ak Du store trofaste Gud , hvor kostcligt dommer Du ! Nu hedder det , at Job har talet ret ; han stulde ikke slippe ud af Luttringsovnen , forend hans Fromhed havde svigtet ham , og han stulde slippe derfra sondcrknust over sig selv , ja blive bestrasset af Gud ! Men sactlidet tenker Gud Herren derpaa . saalidet dommer han Job efter det Onde . som i Fristelsens Tid udbrod af hans svage Kjod . at han nu siger , at hans « Tjener Job har talet ret . " O Du kostelige Gud , ler os dog engang et Lidet at forståa Dig ! Vi anfsre endnu nogle Exempler . Om noget Menneske paa

<h4>1038</h4>Vrede forbande hende ; thi saaledes fremstiller en stig Tildragelse sig altid for et gudfrygtigt Hjerte . O hvilke Tordenslag maa ikte hendes Hjerte have folt den ene Gang efter den anden af fslgende Tanker : « Ingen anden cnd Du er Skylden i , at han nu er tabt ; dct var Dig , forn skulde vogte ham og ikke et Oieblik lade ham komme fra Dig . " Og videre : « Dette har Du for Dine Synders Skyld ! Gud havde ellers kunnet lede det bedre , men han er tret af Dig ; Du er for Dine Synders Skyld uverdig til den store Naade og Mre . forn var Dig given ; nn har Gnd tåget sin tilbage . Du er Skyld i , at Verdens Frelfer nu er tabt ; Gud og alle Mennesker maa fordumme og forbande Dig ; Du er den grueligste Synder , som nogensinde har gaaet paa denne Jord . " See , med saadanne gyscligc Tanter pleier Djevelen at bnldre Hjertet fnldt hos gudfrygtige Mennesker , nåar fsrsi en Forseelse og derpaa en Ulykte er indtruffen ; han tordner og forbander dem uden <b>Maal</b> ug Grendse med de urimeligste Tanter og Forskrekkelser . Og saadant maa Guds Sons <b>Moder</b> gjcnnemgaa , saa at hun nu gjerne tier mcd sin hoie Mre , at hun er Guds Sons <b>Moder</b> , ligesom David gjerne tang med sin <b>vise</b> Reqering , da han saa sig skyldig i halufjerdsindstyve Tusinde Oennesters Dsd for sit Hovmods Styld . Saaledes pleier Gud at gjore med siue bedste Venner . Men hvad den fromme Simeon havde forudsagt Maria med de Ord : « Et Sverd stal gjennemtrenge Din <b>Sjel</b> " , var endnu itte opfyldt . Hvor oste er itte et glsoende Sverd gaaet igjennem dette Moderhjerte , nåar hnn stod i Hoben blandt Jesu Tilhsrere og merkede hans Fjenders store Bitterhet » , morderiske Hensigter og Anstag imod ham ! Naaden borttager itte de rent naturlige Folelser — Kristus selv , bevede i Urtegactrden — Maria har havt et sandt Moderhjerte . Hvad har hnn da ikke folt , da det omsider var kommet derhen , at hendes Son virkelig var ide brolende Lsvers Vold ! Den anden Dag maa hun hore , at de have mishandlet hans Legeme , det Legeme , som hun ligefrct hans Fodsel med saa stor Ombed havde pleiet — de havde « hudflenget " ham Tent , hvad dette var sor hendes moderlige Hjerte ! Omsider er han domt til at henrettes , og hun maa see ham at bere et tungt Korstre til Retterstedet ; thi nu var hun med i Sorgetoget , som Historien viser . Snart seer hun sin Ssn henge paa Korset med Spiger igjennem Hender og Fodder . Han henger paa grove Spiger i Luften . Nu git visselig Sverdet igjennem hendes <b>Sjel</b> . Tenk Dig og glem aldrig , hvad Gnd kan lade sin mest benaadede Ven vaa Jorden gjcnnemgaa ! Og tenk , om vi betragtcde Martcrmanden selv , vor dyrebare Frelser som hengtc paa Svigerne og i sin <b>Sjel</b> solte Helvcdes Kvaler — hvilket stort Exempel ! Mcn dm Lidelse tan intet Mcnncstc forståa ; dog er dct visselig ct stort Exempel paa den samme Guds undcrligc Rcgering , ncmlig at ui ved Lidelser 59 *

<h4>1040</h4>stulle indgaa i Herligheden . Derfor siger ogsaa Apostelen : La » der os med Taalmodighed gjennemlooe den os anviste Kamp og betragte lesum , som , da han lunde havt Glede , led Korset og ågrede ikke Forsmedelsen og sidder nu ved den hoire Side af Guds Throne . Tenk paa ham . forn har lidt en faadan Modstgelfe af Syndere imod sig , saa I ilte trettes i Eders Sind og opgive Haabet . Hvad stulle vi nu lere af alle disse Exempler ? Vi stulle forst tjente nogen Forstand om Livet i Almindelighed . Der bliver visselig ikte blot Sporgsmaal om at have en noget lettere eller tungere Levnetsvei her paa Jorden , men om en stor , evig Udgang af Veien — en stor , evig Salighed eller Fordommelse . Det er det Forste , alle disfe Exempler lere os , at den store , almegtige, hulde Fader , forn i Begyndelsen stabte os i sit Billede og til at nyde Salighed , ikte vil her paa Jorden tildele os vort Gode , men tverimod regerer saaledes , at de , som i Evigheden stulle have himmelsk Glede hos ham , de stulle i denne Verden image paa dct Bittre saa alvorlig , at det vel kan tjene til at sorhoie den himmelske Salighed og derhos at drebe hans og vor selles Fjende , Synden , rense og luttre vor <b>Aand</b> , <b>Sjel</b> og Legeme for det ftore <b>Maal</b> , hvortil de ere bestemte . Dette er den almindelige Forstand , vi sorst bor hente af alle disse Exempler . Men derom have vi forud talt . Det Andet er , at nåar vi see , at de allerhelliastc Mennesker Gnds kjereste Born og Venner , have i hele deres Liv paa Jorden havt en Kjede af Provelfer og Lidelser , saa bor vi styre Vor Utaalmodighed lidet og bcgynoe at nsies med det ringere <b>Maal</b> af Lidelser , han tildeler os . Hvorfor stulde vi have det bedre end disse store Helgene ? Vi vide alle , at Lidelsen for en stor Del beroer paa , hvorledes vi betragte den , og at Utaalmodigheden saaledes ofte legger den tungeste Sten paa Byrden. Du , som soruudrer Dig og klager over Din Lidelse , Du maa da vere et besynderlig udvalgt Menneste , efterdi just Du ikke bsr lide Noget ! Hvorfor stal just Du have det bedre end David , som var « en Mand efter Guds Hjerte ? " Du maa nodvendig have det bedre end Johannes , forn var « den Storste , der var fodt af en <b>Kvinde</b> , " og nodvendig have det bedre end Maria, Guds Sons <b>Moder</b> ! Maastee Du aldrig endnu har havt saadanne Sorger som David , da hans Ben forteredes af Angest, og han ved svere Syndefald var bleven til en almindelig Stam . Du har sikkerlig aldrig lidt , hvad Jomfru Maria led , da hendes dyrebare Barn var tabt . Hvorfor stal just Du nodvendig have det bedre end David og Maria ? Har Du da fortjent at have det saameget bedre end disse Guds Helgcue ? Betenk dette dybt , saa stal Du ikke blot blive tilfreds med Dit <b>Maal</b> af Lidelfe , men maastee begynde at forundres over , om Du itte har altfor lidt og er iblandt dem , forn faa deres Gode , medens de leve .

<h4>1132</h4>terknnse Slangens Hoved , bere Verdens Synder ( Es . 53 , ) og i hvem alle Jordens Slegter stnlde velsignes ? Her , imod disse Aartustndernes li tallige Vidnesbyrd , blive dog mit Hjertes morke Tvivl og Modsigclser som dc for Vinden flyvende Stov og Stråa imod det gamle faste Bjcrg . Gud Herren vere evig Lov og Tak og Pris for sin uudsigelige Gave . Da „ sendte Gud sin Son . " Her er nu sclve Hjertcpunktct i vort Sprog og i hele vor kristelige Tro . Men her er ogsaa det Punkt , som vi aldrig ret kunne faa ind i vort Hjerte . Her er det Punkt , hvorover man blot bnrde anstille Bonner med megen Paakllldelfc , at Gud vilde forbarme sig over os og aabne vort Sind , at vi maatte kunne sec og stnc Noget i de umaadclige Dyb af Guds Naadeunder , da han gav vor arme faldne Slegt sin egen Son . Lad os see , hvorlcdcs Ortene lyte . Det hcddcr : Da sendte Gud sin Son . Merk , der stacer ikke : Da fodtes Guds Son , men : Gud „ fendte " fill Son , fodt af en <b>Kvinde</b> . Det Barn , forn Du i Aanden stuer i Krybben , er ikke et Barn , forn har faaet et saa stort <b>Maal</b> af Guds Aano og Guds Velbehag , at Gud derfor talder det sin Son — nei , anderledes taler Skriften: den siger nemlig saaledes , at det Barn var Guds evige Son . som „ var hos Faderen , ssrend denne Verden blev stabt , " den siger , at i Bethlehem stulde fodes den , „ hvis Udgang er fra fordum , fra Evigheds Dage " ( Mik . 5 , ) at  i Begyndelsen var Ordet , og Ordet var Gud , saa at ved det er Alting stabt , og uten det er intet blevet til af alt det , som er , < ^ og Ordet blev Kjod og bocde iblandt os , og vi saa hans Herlighed som den Enbaarnes af Faderen , fuld af Naade og Sandhed . " Saadant ligger i det „ fendte " — Gud feudtc sin Son , altsaa en Son , som forud var til , men forn nn i Tidens Fylde alene blev fendt til Jorden . Det siger og Kristus selv : „ Saa elskede Gud Verden , at han hengav sin enbaarne Son . " „ Fc > ocr , forklar Din Son mcd den Hcrlighcd , foin jeg havte hos Dig , forend Verden var . " „ leg udgik fra Faderen og kom til Verden; jeg forlader nu Verden og gaacr til Faderen . " Saa siger og Apostelen Johannes : „ Dcri er Glids Kjerlighed til os aabenbaret , at Gud har sendt sin enbaarne Son til Verden , at vi skulle leve ved ham — hali clstcdc os og udsendte sill Son til en Forsoning for vore Synder . " Og ' atter Paulus : „ Det gjorde Gud , da han sendte sin Son i syndigt Kjods Lignelse." See , Saadant stal Dn forståa , nåar ' Du i Nauden sidder vcd Krybben i Bethlehem og ssner det merkverdige Barn , hvis Fodsel bcbndes af hele den himmclste Herskare for fattige Ayrdcr paa Marken og af den undcrfuldc Stjerne for <b>vise</b> Ment , i Osterland . Tak og Lov og VEre og Pris og Styrke vere vor Gud fra Evighed til Evighed for hans undsigclige Gave ! Gud fendte fin Son . Fodt af en <b>Kvinde</b> . Übegribelige Himmclunder ! Guds

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100603016'target=_>Hofacker, Ludwig, 1857, Prædikener paa alle Søn- og Helligdage</a><h4>168</h4>Ere vi Syndere , saa ere vi skyldige til Lovens Forbandelse , saa er Lovens , Guds Helligheds Forbandelse skreven os i Hjertet , saa have vi altsaa kun Frygt og Ekrcrk for Gud i vorr Hjerte , ikke Kjerlighed og Tillid og Haab til ham . Dette maa man erfare , fsrend man kan troe det ; Hjertets Grund maa fsrst oprsres ; Egenkjerlighedens og den kjsdelige Blsdagtigheds bedragerske Billeder , som svsmme paa Hjertets Overstade , maae fsrst vige ; det maa blive <b>Sjelens</b> alvorlige Onske at vende tilbage til Samfundet med Gud ; da fsrst troer og veed man , at Gud er en forferdelig Gud for Synderen, og at en Ekilleveg staaer mellem ham og Mennesket , hvilken ingen Skabning kan gjennembryde . Men endnu Mere ! Vi ville ikke heller komme til ham ; vi have ingen Glede i ham ; vi have ingen Lyst til ham ; vi have andre Guder , og disse tjene og tilbede vi heller , end den levende Gud . O kjere Tilhsrere , betenker dog , hvor langt vi ved Synden ere komne bort fra Gud ; hvor aldeles vi have ladet os forvilde og formsrke og hendrage af det , som ikke er Gud . Den Ene lader sig l endrage af dette , den Anden af hiint ; den Ene gjor dette , den Anden hiint til sin Afgud ; den Ene soger sit Hjertes Glede i sine Marker og Eiendomme , den Anden i sine Penge , den Tredie i sit Haandvcrrk ; en Fjerde elsker sin <b>Hustru</b> , en Femte sine Bsrn mere end Gud og Frelseren ; en Sjette setter sin Glcrde i Fraadseri og Drukkenskab ; en Syvende i 2 Ere hos Menneskene , en Ottende i smukke Kleder , en Niende i mange <b>Kundskaber</b> , en Tiende i sine naturlige Gaver og Anleg , en Ellevte i allehaande Dyd og Mrbarhed , som han mener at have tilegnet sig ; en Tolvte endog i aandelige Gaver , som Jesus har skjenket ham ; og denne sidste Afgudsdyrkelse er den allerfarligste . I disse og andre- Ting fordybe Menneskene sig ; i dem soge de sin Tilfredsstillelse ; i dem behage og speile de sig ; og i Almindelighed har et Menneske ikke blot en enkelt Afgud , men ti og hundrede og endnu flere . See , saa elendige ere vi ; vi kunde ikke komme til Gud , dersom han ikke kom til os ; da maatte vi forblive fortabte og forvildede Faar , fortabte og forvildede paa vore egne Veie , fastbundne og lenkede til Synden , fengslede med Lenker til Msrkets Gjerninger , under Syndens , Djevelens , Dsdens og Helvedes Herrcdsmme . Dette vilde vere vor Lod , dersom han ikke var kommen og endnu kom til os .

<h4>1327</h4>Christus alene kan hjcelpe os . ' Dette bidrog ogsaa til den stcerke Tro hos den cananceiste <b>Qvinde</b> , at hun ikke vid ste no gen anden Hjcelper , end Davids Son . Havde hun vidst nogen anden, da havde hun vel neppe trcengt saa haardt ind paa denne og tvunget ham til Hjcelp . Men nu kunde hun ikke saa snart blive trått i Kampen med ham ; thi hvor stulde hun ellers gaae hen ? Vi derimod i Christenheden kjende mange andre Hjcelpere , end Christum , som kunne frelse os fra Helvede . En Hovedhjcelper , som man i vor Tid ncevner , er en Ting , som man kalder Dyd , om hvilken Ingen ret veed , hvad den er . I de hsiere Ståender har man forudcn Dyden opfundet endnu en anden Hjcelper ; man kalder den <b>Dannelse</b> . Dermed mener man intet Andet , end at der for alle dem , som have en bedre Klcedning paa , ikke gives nogen Helvedes Ild . Endog blot det Navn " Helvedes Ild " er for saadanne dannede Sjcele anstsdeligt og stridende mod den gode Smag ; de mene allerede af den Grund at have Himmelen i sikker Eie . Saaledes har man i hsiere og lavere. Stcrnder endnu mange Hjcelpere . Men Hovedhjcelperen er Letsindigheden eller , for at jeg stal sige det lige ud , Djcevelen . Han vil hjcelpe sine Tjenere og hjcrlpe dem ned i Ssen , som brcender med Ild og Svoul , i den anden De > d . Sandelig , det er paa he > i Tid , at vi gaae til Christum og indstcendig anraabe ham om at frelse os fra Morkets Magt og fra denne onde Tid , i hvilken Djcevelen paa en meer end almindelig Maade driver sit gruelige Uvcesen . Hvo som ikke med fuldt Alvor soger Frelseren , ham vil Verdens <b>Aand</b> opsluge . O Herre , oplad vore Oine og hjcelp os ! Amen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014082608042'target=_>Luther, Martin, 1851, Dr. Morten Luthers udførlige Forklaring over St. Pauli Epistel til de Galater</a><h4>517</h4>terhed , at hende Synden ret , siger jeg , paa den Maade , det forstrcrkker Hier , tet , forer Fok l Fortvivlelse og drcrber dem , Rom . 7 , M 11. derfor at talde saadann ? Lovens Lcrrere Plagefogeder og Tyranner . ' og Plagede Israels Bern med legemlig Tlceldom , saa fore disse ogsaa de arme Sjcrle i en <b>aandelig</b> og derfor langt haardere og utaalellgere Trcrldom ved Lovens <b>Lcrrdom</b> , og gjore det , at de omsider ganste maae fortvivle og fortabes . Saadanne Lovens Lcerere kjende derfor hverken sig selv eller Lovens Kraft ; om de derfor endffjont holdte alle Ordensregler , havde stor Kjcerllghed , gjorde mange gode Gjerninqer , leed der- det dem dog umuligt , at de stulde kunne stille deres Samvittighed tilfreds , nåar de ret forstrcrkkes for Guds Vrede , og ere i Dodsangest; thl Loven lader aldrig af , men den strcekker stedse , anklager Samvittigheden, og stger : Du har endnu ikke fyldestgjort Loven : c . Denne Forstrcrkkelse maa derfor stedse blive jo lcrngere jo storre . Dersom da saadanne Lo- lkte hjceipes ved Troen , saa maae de omsider fortvivle , om nE e - et mcrckligt Exeuipel i den Bog , man kalder Viw « mit , der " " en Eremit , som kort for han dode stal have staaet i hele tre han og uden Afiadelse seet op tll Himmelen ; men da han blev spurgt , hvordoe. for dan g / orde det ? svarede han , at han var bange for Doden . Da nu hans trosiede ham , og sagde , hvorfor han saa vilde frygte for Doden , det kulde han lkke gjore ; thl han havde jo fsrt et helligt og godt Liv ' svarede Jeg har vel fsrt et helligt Levnet , og holdt Guds Bud , men < suds Dom er langt anderledes end Mennestenes Dom . See der , ihvorvel denne havde fort et helligt og ustraffeligt Levnet , saa knnde han dog ikke vcrre rolig l samvittigheden , da han skulde doe ; thi han erindrede sig , at Gud vor Herre domte langt anderledes end Mennestene . Han lod derfor fare oa maatte lade fare al den Tillid , han havde sat til sine gode Gjerninger og Forteneste det kan lkke siaae feil , at hvis han ikke er bleven oprettet ved Kristi HorMtelse , saa har han ganste vist maattet fortvivle . Loven kan derfor lkke gjore andet , end at den aabenbarer kun vor Synd , og bestylderosder er da intet Raad eller Redning mere , men der <b>moder</b> kun idel Jammer f og Elendighed , alle Helgenes Levnet og Marter kan da ikke bjcrlpe os . n " ' Wsaa meget smukt og ligesom i et Forbillede betegnet , da Loven er blev given paa Sinai Bjerg , 2 Mos . 19 og 20 Cap . Der sinder du bestregiven. v " , hvorledes Moses forte Folket ud af Telterne imod Gud Herren , paadet at de srulde hore Gud tale af det Morke og af Skyerne . Der dette steede , sorscrrdedes folket og frygtede sig , og veeg tilbage ; da de dog tilforn havde adet stg forlyde , at de vilde gjore og holde alt , hvad Gnd vor Herre vilde vesale dem ; og der de stode langt borte , sagde de til Mosi : „ Hvo kan laale at ste den forferdelige Ild , og hore den forstrcrkkelige Torden og Basunelyd ? du med os , og vl ville vcrre lydige , og lad Gud ikke tale med os , at vi ikke doe . " 22. Det er da Lovens eget Arbeide , at den forer os nd afTelterne , af vor Leir , det er , af vor Sikkerhed og af vor Fred og Tillid , som vi have til os stlv , og stiller os frem for Guds Aasyn , og aabenbarer os hans Vrede ; da bliver Samvittigheden forsi våer og foler , at den har ikke fyldestgjort Loven , ' tre heller nogenttd kan fyldesigjore den , saa den kan og ikte taale den Guds som Loven aabenbarer . Men nåar Loven saaledes fremstiller os for Gnds Sme , det er , nåar den saaledes forstrækker , anklager og aabeubarer Synden , saa er det umuligt , at vi kan besiaae . Vi maae da forfcrrdes , flye tilbage og raabe : Vi doe , vi doe , lad Herren dog itke tale med os , men tael du med os : c . 2 Z . Enhver derfor , som lcrrer at Troen paa Christum ikke retfcrrdiggjor , medmindre man tillige holder Loven , han gjor Christum til en Syndens Tjener , det er , han gjor ham til en Lovens Lcrrer , der lcrrer det selvsamme som Moses , og mtet andet . Saa kan da Christus ikke vcrre nogen Frelser og Naadens Glver , men han maa vcrre en grum Tyran , der krcrver kun lutter umulige

<h4>751</h4>holdt dem ogsaa med al Flid , saa frelse de dog ingen fra Forbandelsen for Gud . Kongeriget i Babylon var stiftet af Gud og givet Kongerne , havde ogsaa en meget stjon Lov og Net , og alle Folk vare befalede at vcrre disse Love og Rette lydige og underdanige ; dog kunde Lydighed og Underdanighed mod dlsse Love ikte frelse noget Menneste fra Guds Lovs Forbandelse . Saa ere vl nuomstunder de keiserlige Love underdanige , dog alligevel blive vi derved lkke retfcrrdige for Gud , thi vi have her med en anden Sag at bestille . <b>Aandelig</b> 4. At jeg formaner saa flittigt , at man vel og egentlig ber liere at gjere U Forstjel l denne Post , det gjor jeg ikke forgjeves ; thi det er meget nyttigt , at fcrrdiahed " " " vel veed denne Forstjel , ihvorvel man finder faa , som ret agte og forstaae ' den . Tilmed er det ogsaa meget snart og lettelig gjort , at man kan blande den hlmmelste Retfærdighed med den verdslige Retfærdighed . I verdslig Retfærdighed maa og bor det da vcere , at man stal see paa Lovene og Gjerningerne; men i den aandelige , guddommelige og himmelske Retfærdighed maae vi reent scrttealle Love og Gjerninger tilside , og alene see paa Abrahams Vel ? signelse og Forjættelse ; hvilken Forjættelse foreholder os Christum , som den vi blive velsignede , det er , gives Naade og Salighed ved ; saa at den , som vil blive retfcrrdig for Gud , og vil blive hans Arving i Himmerig , han maa alene see paa den Naade og Velsignelse , som os tilbydes og gives i Christo , uden nogen Lov og Gjerninger , ligesom den blev forjcrttet Abraham , og troet af ham . Hvad St . 9. Heraf see vl , hvad St . Paulus mener , naar han taler saa haanligt og mener der- foragteligt om Loven , og siger : „ De , som ere af Lovens Gjerninger , de ere medaehan " nder Forbandelsen ; " nemlig at han ikke taler om Love og Nette , som henhore taler saa ttl det verdslige Regimente , men han handler om , hvordan vi kan blive rethaanligt fcrrdige for Gud : om vi kan undgaae Forbandelsen og faae Velsignelsen ved og forag- Loven ? Derpaa svarer han da : „ De , som ere af Lovens Gjerninger , de ere Loven ° " " " d " Forbandelsen . " Af Aarsag , fordi af Loven kommer kun Syndens og Guds Vredes Ertjendelse . Det er derfor et meget fast Argnment ; thi er det sandt , at man bor alene vente og faae Velsignelsen ved Christum , saa maa det derimod ogsaa ganste vist vcere sandt , at man ikke kan faae den ved Loven ; iligemaade er Velsignelsen forjcrttet den troende Abraham ikke alene i Lovens Tid , men og lcenge tilforn , nemlig 430 Aar for Loven , og det er regnet ham til Retfcerdighed , at han troede Gud . Skal vi nu blive deelagtige i Velsignelsen, saa maae vi antage den i Troen . Det er alene Forst / ellen : Abraham troede paa Christum , som endda stulde komme og tilfore Velsignelsen , men vi troe , at han allerede er kommen ; vi blive da retfcerdige ved Troen , ligesom Abraham er bleven retfcrrdig ; men de , som ere af Lovens Gjerninger , og ville derved faae Velsignelsen , de ere under Forbandelsen . 6. Men dette kan og vil Paven med sine Afguder , Cardinaler , Biskopper : c . ikketaale ; vi bor derfor ikte tie hertil ; thi vi ere jo under vor Saligheds Forliis pligtige til at bekjende Sandheden . Derfor sige vi , at Pavedommet er forbandet med alle sine Love ; iligemaade , at Keiserens Lov og Net ogsaa er forbandet . Thi efter St . Pauli Mening er alt det forbandet , som er udenfor Abrahams Tro og Modstan- Forjcettelse . Men naar vore Modstandere hore det , da forvende de vore Ord , derne be- forklare dem vrangeligt og foregive , at vi lcrre , man stal ikke agte øvrigheden ; ' for at vi gjore intet andet med vor <b>Lcrrdom</b> , end at vi kun stifte Opror mod Keiseren , gjore fordomme alle Love , og ville kuldkaste og tilintetgjore al Orden og Regimente , verdsligt 7. Men de gjore os dermed storste Uret , thi vi tale med Forstjel og For , Regimente sigtighed om legemlig og <b>aandelig</b> Velsignelse , og sige , at Keiseren er velsignet , tantet , dog med legemlig Velsignelse . Thi at man har Kongeriger , Herredomme , Lov , Verdslig Ret og verdslige Skikke ; saa og , at man har <b>Hustru</b> , Born , Huus , Hjem , leaemlia " : c . det er sandelig en Velsignelse , thi alt dette og deslige er idel Velsignet- Ting , som ere stable og givne af Gud ; men ved disse legemlige Velse. ' signelser , som de der ere timelige og endelige , blive vi ikke frelste fra den evige Forbandelse . Vi fordomme derfor ikke verdslig og keiserlig Ret , ikke heller

<h4>950</h4>hvor hoi- nsdent at tale saa ofte og vidtloftigt derom ; men jeg veed , at der ligger aldeles ligt det er scor Magt paa denne Underretning og rette Forstand paa Loven . Jeg taler fornodent , derfor ikke uden Aarsag saa vidtloftigt derom ; thi jeg vilde gjerne , at enhver forståa « Christen havde fuldkommen <b>vis</b> Underretning derom , nemlig at Loven har en Loven . ganske anden , ja en ganske stridig Virkning imod den Folk mene , da de af Naturen allesammen have den stadelige Indbildning og Grille , at de holde for , den gjor os retfærdige ; den , som siger andet , han maa vcrre en Kjcrtter . Jeg har da gjort Mit , og af St . Paulo troligen viisi , hvad dens rette Embede er , hvorlcenge den stal vare : c . men som jeg seer , da ere der kun faa af dem , som ere Ordets Tjenere , der vide det . Jeg frygter sandelig derfor , at Lærdommen Morkhed i efter min Tid vil blive undertrykt igjen ; tbi Verden maa fsr den yderste Dag Verden for blive opfyldt med gruelige og forstrcetkelige Msrkheder og Vildfarelsers . den yderste 13. Derfor , hvo som kan , han fatte det , at Loven efter den chrisielige Theologi , og nåar den er i sin rette Brug og Embede , gjor ikke retfærdig , men udretter det , som er ganske tvcertimos ; nemlig den aabner os Amene , og viser os , hvem vi ere , strcekker os , og ikke alene aabenbarer os Synden og Gnds Vrede , men den gjsr endogsaa Synden sisrre og mcegtigere , saa at , om den tilforn uden Anfegtning syntes let og ringe , da bliver den stor og svar , nåar Lovens Glands falder paa , saa at Mennesket begynder da at blive Loven Loven gjor fiendsk , derhos ogsaa ret at hade Gud , som har givet Loven . Det er jo da os vcrrre , ikke at blive retfærdig af Loven , hvilket Naturen selv maa tilstaae , men det hvorstulde er at have syndet dobbelt imod Loven ; fsrst , at man har en ond Villie dertil , fwrdiqe . bliver man den saa fiendst og vred , saa man gjerne snstede , der var siet ingen Lov til , ikte heller Gud , som har givet den , endstjsnt han er den allermilde ; ste Herre og hans Lov er retfærdig og hellig . 14. Men kan man vel hsiligere og grueligere bespotte Gud ? Kan man vel forsynde sig groveligere imod ham , end nåar man hader ham , er hans Menne- kov fiendst , og ikke gider hsrt den , da den dog er god og hellig ? Thi Histo ; sketsHjerte rien 2 Mos . 19.20 . viser jo tlarligen , at Israels Folk ikte kunde taale at hsre kan ikke den allerbedste Lov og de allerhelligste Ord , som deri indeholdes , uden en Mid ; taale thi er det ikke deilige og trssielige Ord : „ Jeg er Herren din Gud , som Lov Mont udfsrte dig af TEgypten " : c . 2 Mos . 20 , 2. „ Dn stal ikke have fremmede den tilsiger Guder for mig " : c . V . 3. „ Som gjsr Miskundhed paa mange Tusinde " : c , V . 6. alt Godt . Item : „ A3r din Fader og din <b>Moder</b> , at det kan gaae dig vel og du maa lcrnge leve paa Jorden " V . 12. Denne allersisrste og hsieste Guds Viisdom , som er den stjsnnesie , bedste og deiligste <b>Lcrrdom</b> , den kan de ikte taale , men sige : „ Kjcrre , lad Herren ikke tale med os , paadet at vi ikke dse ; men tal du heller med os . "

<h4>1180</h4>Ord , for at faae Tllhcrng . St . Paulus siger Rom . 16,18 . at de tjene deres egen Vug , og ved sede og store Ord forvirre de uskyldige Hjerter . Det fsrste de snige sig md , stadfeste de hsit og dyrt , at de imet soge andet , end at de alene tan fremme Guds 3 Ere og Menneskenes Salighed ; saaog at de drives as Aanden til at lcere den rene Sandhed ret , saasom de ste , hvor ilde det arme solt b Uver forssmt , eller at Guds Ord ikke bliver prcrdiket af andre , paadet at de Udvalgte kan ved dem komme ud af Vildfarelser , og fores til det rette og sande Lys , og tll en <b>vis</b> <b>Kundskab</b> om Sandheden ; derhos tilsige de oqsaa dem , som antage deres <b>Lcrrdom</b> , at de ganffe vist stal blive salige . Og under dette af Gudfrygtighed og Faareklcrdernes Skjul gjore de glubende Ulve L. hr lstnememgheder overmande stor Skade , hvis Lcererne og Sjcrlessrgerne ikte ere aawaagne og forsigtige , og alvorligen gjsre Modstand . 2. Her kunde de Galater vel sagt : Hvorfor taler du saa haardt paa vore sordt de ere nidkjære for os , og gribe sig det an foros ; de gjore det jo dog as gudellg Nidkjcerhed og af lutter Kjcrrlighed ; hvorledes kommer det sig da , at du forarger dig derover ? Derpaa svarer St . Paulus og siger : Det er vel sandt , at de ere nidkjcrre for eder , men itte vel . Sacra- 3. Ligesaa maae vi nuomstunder hore af Sacramentererne , at de bestylde meuterere os sor , vl ere altfor stive og halsstarrige , ndrydde al Kjcrrlighed og Samdrægtighed af de christne Menigheder , idet at vi tale imod deres <b>Lcerdom</b> om Nadveren , og ville ikke holde den for ret ; de mene , det var langt bedre , at vl saae lgjennem Fingre dermed og gav dem noget efter , fornemmelig da der Ute ligger stor Magt paa , end at vi for denne ene Artikkels Skyld , som dog ikke stal vcere nogen af de fornemste , ville gjore saa stor Uenighed , Splid og Strid t Christenheden , da de dog ellers ikke ere os imod i nogen anden Ar ? tlktel af den christelige <b>Lcerdom</b> , uden alene i denne ene om Nadveren . Der . - ttl svarer jeg saaledes : Forbandet vcere den Kjcrrlighed og Enighed , for hvis Skyld man stal antage en vrang Mening i Guds Ord . A " ad Nid- 4. Men at vi stal komme til Sagen igjen : Da foregav de falste Apostler , rl « lyeoer.atde hsiligen elstede de Galater , og vare nidkjære for dem med en ret gndelig Nldkjcrrhed . Nidkjcerhed er da en vred Kjcrrlighed , eller et venligt og godt Had . Som naar jeg elsker nogen , og seer , at han gjor ilde , da gjor det mig hjertelig ondt , jeg bliver vred og fortrydelig derover , og straffer ham , ikke af Flendstab , men af Kjcerlighed ; det kaldes da Nidkjcerhed , det er , en vred Kærlighed, eller et barmhjertigt Had eller Vrede . Saaledes er en A3gtemand mdkjcer for sin <b>Hustru</b> , en Fader for sin Ssn , en Broder for den anden ; det er : den ene elster og mener den anden ret oprigtigt , dog saaledes , at man hader de Laster , der kan vcere hos en , straffer dem , og sircrber at rette dem ; saaledes var St . Paulus nidkjcer for de Galater . Saadan Nidkjcerhed foregav de falste Apostler ogsaa for de Galater , dog det var ikke deres Alvor . Under hvad 5. St . Paulus advarer derfor de Galater med disse Ord , som han vilde Ass ? / « - Lcerere anstille sig vel , som de havde stor inderlig Kjcrrlighed til rere pleie eder , de ere ogsaa nidkjcere for eder med stor Alvorlighed , som de meente det atbedrage ganste oprigtigt ; men det er ikke en god Nidkjcerhed ; og ret under dette Skin de Enfol- bedrage de eder , da de stille siq an , som de elstede eder hoiligen , og bare hjerdlge. telig Omhu for eder . St . Paulus formaner da her de Galater og os alle , at God og vi stal gjsre Forstjel paa en god og en ond Nidkjcerhed ; en god Nidkjcerhed er A ' b N ' d - hsiligen at rose , men ikke en ond . Jeg er da ligesaavel nidkjcer for eder , siger St . icrryev . Paulus , som de ere ; men dommer nu , hvilken Nidkjcerhed der er den bedste , enten min etter deres ; saaog hvilken der er god og gudelig , eller ond og kjedelig . I tsr derfor ikke tage eder det saa ncrr , at de rose sig af stor Nidkjcerhed for eder , saasom De ville udelukke eder fra mig , at I stulle vcere nidkjcere efter dem . FalsteLcr- 6. Som han vilde sige saa meget : Vel er det sandt , at de ere optcendte af Kjcerlighed til eder , og ere meget nidkjcere for eder ; men de soge dermed ' intet andet , end at I iqjen stal vcere nidkjcere for dem , blive affceldige fra mig ,

<h4>1190</h4>gode Navn og Nygte , og udtolke til det vcrrsie alt hvad godt og ret de have lcrrt . De falske 21. Saadan daarlig Nidtjcrrhed var der og hos de falske Aposiler ; de gjorde Apostlers derfor Partier , ilke alene i Galatien , men og paa alle andre Stceder , hvor daarlige Panlns og andre Aposiler havde prcrdiket , som de allevegne klage over i Nidkar- Paa diss ? Partier der da siden fulgte utallig mange Forar ? gelstr og Forvirrelser ; „ thi Djcrvelen er , " som Christus siger Joh . 8 , „ baade en Logner og en Morder ; " han pleier derfor ved sine Tjenere ikke alene at forvirre Samvittighederne med falsk <b>Lærdom</b> , men ogsaa opvcetke Usamdrcrgtighed , Opror , Krig , Mord , : c . . . . « 22. Saadanne Nidkjcerhedsaander haves l disse Ttder mange af t Tydstland, hvilke ere ganffe forgabede i sig selv , og ville holdes for gudfrygtige , siedelige , lcerde , taalmodige : c . Folk , da de dog i sig selv og isandhed ere nogle Der ere hidsige og arrige Hyklere , onde Orme og glubende Ulve , som med dette deres mange i Hykleri ikte soge eller strcebe efter andet , end at de ganffe kan undertrykke voreNder , som have den rene <b>Lcrrdom</b> og bekjende ' sig til den , gjore den forhadt hos ! n " f ? lif " enhver , at alle stal kun see paa dem alene , cere dem , prise og ophole deres Nidkar- <b>Lærdom</b> ; men fordi de foragte andre og have tun hoie Tanker om sig selv , hed . saa er det et vist Tegn , at de soge hverken Chrljw2Ere eller Menneskenes Salighed , men at de ville ceres og ansees af enhver ; hvoraf da intet andet kan folge , end Uenighed , Tvedragt , Nid , Had , Vrede , Pam , Trcrtte , Splid , Opror , : e . Hvad kan da vi dertil ? hvad kan vi gjore derved ? Vl kan des ? vcrrre ligesaa lidet afvcrrge det , som St . Paulus . Dog alligevel har han vundet nogle og fort dem tilrette igjen , som vilde vcere hans Formaning ly ? dige . Saa haaber jeg ogsaa , at nogle af dem , der ere blevne forforte af Partierne , stal ved vore Formaninger ogsaa blive hjulpne tilrette lgjen . W . iB . Men det er godt at vcere nidkjcer altid i det Gode , og ikke alene , naar jeg er ncervcerende hos eder . Som han vilde sige : Jeg har roset eder , fordi I vare nidkjoere for mig , og elskede mig saa hoiligen , der jeg prædikede Evangelium for eder i mm Skrøbelighed efter Kjodet ; men ligesom I da elstede ung ncervcerende og vare md ? kjwre for mig , saa stulle I uu elste mig i min Fraværelse og vssre mdhcere for mig , som om jeg endnn var hos eder , og aldrig havde vcrret stilt fra sandelig eder . Thi er jeg endstjont borte fra eder efter Legemet , saa dog end ? og kjedelig „ „ min <b>Laerdom</b> hos eder , og da I derved have faaet den Helligaand , saa Nldkjcer- I billigen beholde og tage vare paa den , forestillende eder , ligesom St . Paulus endnu stedse var ncervcerende , da I have hans <b>Lcrrdom</b> hos eder . laster derfor ikke eders Nidkjcrrhed , men jeg roser den , dog saavldt der er en guddommelig eller <b>aandelig</b> , og ikke en kjodellg Nidtjoerhed ; men den gud ? dommelige Nidkærhed er alene retstaffen , thi den er mdtjcer for noget Godt , men det er den tjodelige ikke . 2 St . Paulus roser da de Salaters Nidkjcrrhed , paadet han kan stille deres Sind tilfreds , at de stal holde ham hans haarde og starpe Tiltale tilgode . Som han vilde sige : Holder mig min Skjelden tilgode , thi den jreer ikke af et ondt og vredagtigt , men af et bekymret Hjerte , som er omhyggeligt for eder . Hvorledes 3 Her have vi atter et Exempel , hvorledes en god Hyrde stal bcrreOmYu c > , god for sine Faar , og vide at stikke sig paa alle Kanter , at han kan holde dem Cicrlesor- salige <b>Lcrrdom</b> , og forsvare dem mod Forforerne , enten det da « reer eller gode Ord , med Beden , eller hvordan det kan . W i 9. Mine Bsrnlille , hvilke jeg atter foder med Smerte , mdtll Christus faaer sin Skikkelse i eder . At han hidindtil har givet saa mange gode og sode Ord , og endnu kalder dem sine Bornlille , hvilke han atter som en <b>Moder</b> foder med Smerte , det gaaer altsammen ud paa , at han gjerne vilde stille de Galater tilfreds , og bevoegc

<h4>1222</h4>forfolges og fordommes som Trcelle , ja som Kjcettere og Djcevels Born , saa ere de dog i Aanden de Frie , dem St . Paulus her taler om , og Sarcr Born , som lkke alene kan dsmme om al <b>Lcerdom</b> og Gjerninger , men ogsaa ere rette Undertvingere og Seiervindere over Helvedes Porte . Christen- 9. Propheten Esaias bekjender da , at den christne Kirke er bedrsvet og sorrige bedrsuct havde sandelig ellers ikke trostet og opmuntret den til at vcore glad . ufrugtbar . H " u tllsiaaer ogsaa , at den er ganske ufrugtbar for Verden ; hvor stulde han ellers kalde den ufrugtbar , som ikke foder og som er forladt ? Men dog ikke desmindre siger han , at den er frugtbar for Gud , hvorfor han ogsaa befaler den at gloede sig , og siger Cap . „ Fryd dig , du Ufrugtsommelige , som itke fsdte ; glcrd dig med Fryd og våer glad , som ikke var i Barnsnod ; thi den Eenliges Born ere fiere end hendes , som haver Mand , sagde Herren . " Som han vilde sige : det er ganste vist , du er eenlig og ufrugtbar , har ikke Loven til Mand , hvorfor dn ikke heller kan have Born ; men vcer ved et frit Mod , thi endstjont du vel ikke haver Loven til Mand , og , som en mandvoxen lom / fru , er forladt og eenlig , ( thi en Enke vil han ikke kalde den , som vel en Brndgom , hvis den ikke var forladt af ham , eller han ikke var ihjelsiaget for den ) . Endstjont du , siger jeg , er eenlig og forladt af Loven , som Man / den , og har ikke nogen AZgtepagt med Loven , saa stal du ikke desmindre vcere <b>Moder</b> til ntallig mange Born . Naadens 10. Christenheden eller det nye Testamentes Folk er da , saavidt Samvittig / « olr . heden angaaer , aldeles uden Lov , hvorudover det synes for Verden , som det var ganste forladt og eenligt . Men lad tun Verden lcengenok sige , at det er uden Lov og Gjerninger og ufrugtbart , saa er det dog meget frugtbart for Gud og foder utallig mange Born , derhos saadanne Born , som ere frie . Men hvorved ? San / delig ikke ved den Mand , der stulde vcrre Lov , men ved Naadens Ord og Christi <b>Aand</b> , som gives / ormedelsi Evangelium , undfanger , foder og avler det Born , : c . Forstjel 11. St . Paulus viser da ganste tydeligen med denne Allegorie , hvad For / ' " ^ " " ^ stjel der er imellem Loven og Evangelium . Forst i det , at ' han kalder det Testamente „ Agar / ' og det nye „ Sara . " Dernest i det han gjor ' den ene til en „ Tjenestepige , " og den anden til en „ fri <b>Qvinde</b> . " Saaog nåar han siger , at den , som har Mand og er frugtbar , bliver strobelig og ufrugtbar , og udsiodes af Huset med hendes Born , men den Uftngtbare og Eenlige derimod bliver frugtbar , og fsder utallig mange Born , som ere Ar / vinger . Det er den rette Forstjel , hvorpaa man egemlig stal kjende Lovens Folk fra Troens Folk : Troens Folk viser St . Panlus af Esaia har ikke Lo / ven til Mand , traller itke , er ilke heller fodt af den <b>Moder</b> Jerusalem , som nu er ; men det har Forjættelsen , er frit , og fsdes af den frie <b>Qvinde</b> Vara . Troens 12. Apostelen gjor da Forstjel paa det nye Testamentes aandelige Folk og Folk . Lovens Folk , idet han siger : „ Men det Jerusalem , som er oventil , er den frie <b>Qvinde</b> , som er alles vores <b>Moder</b> ; " det er : vi ere ikte Agars Born , som har Mand , men Sara ? Born , som er fri , og veed ikke af nogen Lov . Han scetter da Troens Folk langt fra Loven og over den . Men ere de da udenfor og over Loven , saa tan de jo ikte blive retfærdige af Loven , men maae blive retfærdige ved den aandelige Fodsel alene , som da er intet andet end Troen . Men ligesom Naadens Folk ikte har eller kan have nogen Lov , saa har og Lovens Folk ingen Naade , og kan itke heller have den . Thi det er ikke mu / ligt , at disse to stridige Ting , Naaden og Loven , kan forenes og bestaae med hverandre ; det maa derfor endelig vcere eet af Delene : enten at vi blive ret / Lovens fceroige af Troen alene , og lade Lovens Retfærdighed fare , eller og vi maae F ? lk er u - blive retfcrrdige af Loven , og lade Naaden eller Troens Retfærdighed fare . Naadens en sijcendig og ulyksalig Omstiftelse , nåar man slipper Naa / Folk er den , og holder ved Loven ; men derimod er det en nyttig og salig Omstiftelse , uden Lov . nåar vi lade Loven fare , og holde fast ved Naaden . 13. Da vi see , at St . Paulus har gjort sin yderste Flid , saa gjore vi ogsaa

<h4>1225</h4>al vor Flid til , at vi ret noie og tydeligt kan <b>vise</b> , hvad Forstjel der er imellem Loven og Evangelium . Det er vel en let Sag og ganste klart , saavidt Ordene angaaer ; thi hvo stnlde vel vssre saa taabelig , at han ikke stnlde kunne ste , at Agar er ikte Sara , og at Sara derimod ikke er Agar ? Saaog , at Ismael er Forstjel og har ikte det , som Isaak er og har ? Det er ikke nogen stor Knnst at kjende » mellem det ene fra det andet . Men nåar vi ret ere i Beoengstelse og Dodsnsd , da ° 5 vor Samvittighed stal gaae i Strid mod Gnds Lov , saa er det meget tnngt ' at kunne ret med Sandhed af Hjertet alvorligen sige : Jeg er ikke Agars Son , men Sarce ; det er : Loven kommer mig intet ved , saasom Sara er min Mo , - der , der ikke foder Trcelle , men knn dem , som ere Frie og Arvinger . 14. St . Paulns bar da med dette Propheten Esaice Sprog beviist , at Sara , det er , Christenheden , er den rette <b>Moder</b> , hvis Born ere lutter Frie og Ar ? vinger ; derimod at Agar , det er , Synagogen , foder vel mange Born , men de ere dog knnTrcrlle og udsisdes afHuset . Men da dette Sprog ogsaa tillige handler om den christelige Frihed , og hvordan Loven er afskaffet , saa er det vel vcerd , at man ret med Flid overveier det . Thi ligesom det er den vigtigste og fornemste Artikkel i den christelige <b>Lnrdom</b> , at vi vlde , vi blive retfcerdige og salige formedelst Christum alene , saa ligger der ogsaa meget hoit Magt paa , at vi ret vide og forstaae , hvorledes Loven er asstaffet og reent tilsidesat med sit Embede , Loven er saa den ikke kan anklage og fordomme dem , som troe paa Christum ; derved bestyr ? afskaffet , kes vor <b>Lcerdom</b> om Troen . Derhos stal vi ogsaa have Trost deraf for vor Sam ? vittighed , fornemmelig nåar vi komme i store Anfegtnmger og i Dodsnod . 15. Jeg har forhen talt ofte og udforligt derom , og siger det nn atter igjen , thi der kan aldrig siges formeget om dette , at en Christen , som i Troen fatter og antager Christum Verdens Frelser , han er ikke nogen Lov nndergiven , men er ganste fri fra Loven , saa at den ikte kan Engste eller fordomme ham . Det viser Esaias i det Sprog , som St . Paulus her anfsrer , nåar han siger : „ Fryd dig , du Ufrugtbare , du , som ikke foder " : c . det er da fuldt af Trost og vel at agte . Esaias er fremfvr alle andre Propheter meget smuk deri , at han ofte For de i sin Prophetie spaaer saa herligen og tydeligen om Cbristo , hvilken Konge han Troende er : c . men her i dette Sprog saavelsom det hele 54. Cap . taler han om Christi Rige , som er den hellige Chrisienhed , viser , at den for Verden er ufrugtbar , asenet » eenlig og forladt , og maa bcere det hellige Kors , hvorfor han troster den , at den skal have fiere Born end den , som haver Mand : c . 16. Efter Loven ere de Ufrugtbare foragtede og forbandede ; men den Hellig ? Christenaand vender denne Dom om , og siger , at de Ufrugtbare ere at rose og velsig ? heden synede, men derimod de Frugtbare , som fode , ere forbandede , nåar ban siger : for „ Fryd dig , du Ufrugtbares , thi den Eenlige haver fiere Born end den , som haver Mand . " Omenostjsnt derfor Sara eller Chrisienheden synes for For ? nuften at vcere eenlig og ufrugtbar , fordi den bygger ikke hverken paa Loven eller Gjerningerne , saa er den dog for Troens Skyld , hvilken den lcrrer og be ? kjender , en frugtbar <b>Moder</b> for Gud til atallig mange Born , som Propheten bevidner , nåar han siger : „ Thi den Eenlige haver fiere Born end den , som haver Mand . " Derimod synes det for Verden , at Agar er meget frngtbar og foder Born nden Ophor . Thi de , som lEre Loven , have langt fiere Disciple end de , som forkynde Evangelium , thi det er et Ord om Korset , I C0r.1 , 18. Men dog er det altsammen forgjeves , og ingen af de Born blive ved ; thi Tjenestepigens Born blive tilligemed Moderen ndsiodte , og stal ikke arve med den frie Qvindes Born , som St . Paulus siden vil <b>vise</b> os . 17. Ere vi nu den frie Qvindes Born , saa er Loven , som vor gamle Maud , Lovens afstaffet , som der staaer Rom . 7,3 . Thi saalcrnge den herskede over os , da var Folk gjore det umuligt , « tvi kunde have avlet Born i Aanden , som have kjendt Naaden , sig stor men vi maatte alle blevet Trcelle tilligemed dem . Naar Loven herster over " ' " " ge . Menneskene , da ere de vel ikke orkeslost , men arbeide stoerkt , og gjore sig hjer ? Mig megen Umage , de bcrre Dagens Byrde og Hede , fode og avle mange

<h4>1262</h4>frie Qvindes . hermed beslutter St . Paulus sin Allegorie om Christenheden eller Kirken , Tjencstcsom er ufrugtbar , og Lovens Folk , som er frugtbart , og siger : Vi qvmdcn ere ikke Tjenesteqvindens Born , det er , vi ere ikte under Loven , som soder Born til Trceldom , det er , som cengster , anklager dem og kommer dem til at for- tvivle , men vi ere befriede derfra ved Chrisium , hvorfor den og hverken kan Engste eller fordomme os : c . hvorom tilforn vidtloftigt er handlet . Men stjont det vel kan ffee , at Tjenesteqvindens Born en Tidlang forfolge os , saa er dog det vor Trost , at de omsider blive udstsdte i det yderste Morte , og vi maae da beholde Arven , der tilkommer os som den frie O.vindes Born . 2. St . Paulus har da , som vi nu have hort , af de Ord : „ Tjenesteqvinden " Lov / , og „ den frie <b>Qvinde</b> , " taget Anledning til at forkaste Lovens Retfærdighed , Troens og derimod krafteligt stadfeste Troens Retfærdighed , som erlanges ikke ved Gjer- ninger eller Lov , men alene ved Troen paa Christum ; fornemmelig har han holdt sig til det Ord : „ den frie <b>Qvinde</b> , " hvilket han forer sig vel til Nytte , stjcerper og indprenter det vel , som vi og i Begyndelsen af det folgende Capitel fremdeles ville faae at hore , da han deraf ydermere tager Anledning til at tale om den christelige Frihed , hvilken det ogsaa hoiligen ligger Magt paa , at den rettelige lceres i Chrisienheden , fornemmelig da det forbandede Pavedom ganste har fordunklet den rette Mening om den christelige Frihed , og derimod fort den arme Chrisienheo i en ynkelig og stjcendig Samvittighedstrceldom . 3. Denne Frihed , som Chrisius saa dyrt har forhvervet os , er os i denne Den Ar- Tid en stor Bestjermelse og et stcerkt Skjold , hvormed vi bestjerme og forsvare tikke ! om os mod Pavens Tyranni . Hvorfor ogsaa denne Artikkel om den christelige den chri- Frih ed med al Flid bor lnres og agtes ; forst , paadet at den anden Artikkel , som lcerer , hvorved man jkal blive retfcerdig for Gud , derved kan stadfæstes . Derncest ogsaa paadet de arme , becengstede og forvirrede Samvittigheder derved kan hjcelpes tilrette , faae en stadig Trost og <b>vis</b> Underretning imod det meget Onde og de mangfoldige Forargelser , som DjEvelen af Had til Ordet opvcekter , da Modstanderne give Evangelio , eller , som de bespotteligt kalde det , den nye <b>Lcerdom</b> Skyld for det altsammen . Men den christelige Frihed er da en <b>aandelig</b> Ting , som et t ' jodeligt Menneste ikke forsiaaer sig paa . Det falder dem tungt , som endog have begyndt i Aanden , og have nok at lcere derpaa for ret at agte og ansee den . Fornuften mener det er en let Sag ; dersom Aanden derfor ikke gjor den ret stor og dyrebar , da bliver den knn ringe aatet , ja reent foragtet .

<h4>1372</h4>heit om de daarlige Gjerninger , som de selv , uden , ja tvcertimod Gnds Befaling Kjcrrlighave udvalgt , saasom at ste suurt , hcrngemed Hovedet , leve uden VEgtestab , cede heden er Vand og Brsd , boe iOrken , bcere graae Kjortler og deslige : saa er det et vist Tegn til , at de holde Kærligheden for siet intet , da den dog er en Sol for alle Gjer- Gjerninger . Saa übegribelig og overmaade stor er den menneffelige Fornnfts « inger . Blindhed , at den ei alene kan ikke ret skjonne og demme om Troens <b>Lærdom</b> , men end itke heller om de Gjerninger , som Gud har befalet og har Behagelighed til . 19. Vi have derfor nok at bestille , ikke alene med at dcempe vort Hjertes eget Paafund og vildfarende Tanker , dem vi af Naturen heller folge end Gnds Ord , ihvorvel det viser os den rette Vei til Himmelen , og vore egne Tanker fere os til Helvede , men og med at tage Masten af de selvudvalgte Gjerninger , og ikke lade os bedrage eller forfore af deres falske og bedragelige Skin ; ogsaa loere at agte de Gjerninger ret dyrebare , som enhver gjor i sit Kald og Embede , efter Guds Befaling , om de endstjsnt efter det udvortes synes ringe og foragtelige ; men derimod ikke agte de Gjerninger , som Menneskene selv udvcelge uden Guds Befaling , de maae end synes saa hellige , store , herlige og dyrebare , som de ville . 20. Jeg har andetsteds vidtloftigen og tilstrcekkeligen forhandlet og forklar Hvordan ret det Bud : „ elst din Nceste som dig selv , " hvorfor jeg her kun saa korteligt man paavil rere derom . Det bestaaer kun af faa , men meget vigtige og fyndige Ord : „ elst din Nceste som dig selv ; " man kan ikke give nogen et bedre , vissere og neiere Exempel eller Monster paa , hvordan han stal elste sin Nceste , end naar Bud : man siger : han stal elste ham , som han elster sig selv . Ikke heller kan man be . ' elst din sidde nogen heiere og cedlere Dyd end Kjcerligheden ; og denne Dyd kan ikke vises Nafte som imod noget bedre eller cedlere end imod Ncrsten ; det er derfor noget aldeles heit og stlv . herligt , som lceres i dette Bnd . Kjcerligheden i sig selv er den cedleste Dyd , Ncesien, som den stal bevises imod , er det fornemste Creatur i Verden , og man kunde intet bedre Exempel faae at rette Kjcerligheden efter end det : som dig selv . 21. Naar du da vil vide , hvordan du stal elste din Nceste , og derpaa vil Hvordan have et ret tydeligt og vist Exempel , saa betcenk vel , hvor kjcer du har dig selv , man Ml og da stal du besinde , at du elster dig saa heit , saa du vilde gjerne man stulde AM Ncehjcrlpe og undscette dig naar du var i Nod og Angest , ei alene saavidt Men . ' nestene med Liv og Gods kunde formaae , men og saavidt alle stable Ting kunde hjcelpe . Du har da ikke nogen Bog behov for deraf at lcere , hvordan du stal elste din Nceste , thi dn har den allerbedste Bog i dit eget Hjerte , deri finder du beskrevet alt hvad alle Love kan lcere dig ; ikke heller har du nogen Doctor eller Lceremesier behov dertil ; sporg ikknn dit eget Hjerte , det stal vel sige dig , at du stal elste din Nceste som dig selv . 22. Kjcerligheden , naar den ikke er falst , er den da en Dyd , som er villig Hvad og rede til at tjene ikke alene med Mund og Haand , men ogsaa med Liv og Kjcrrlig- Blod . Den lader sig ikke bevcege til at tjene derved , at man forst har tjent hed er for den og gjort vel imod den ; ikke heller lader den sig afholde ved det , at man " ' forst har gjort den ondt og imod . Man kan herpaa gjore sig denne Lignelse : En <b>Moder</b> rygter og foder sit Barn , kjedes ikke ved nogen Umage eller An beide , ja jo mere ureent og sygt Barnet er , jo siorre Umage og Omsorg har hun for Barnet . Barnet fortjener ikke siigt , ja det gjer hende kun Uro . Hvoraf kommer det da , at hun ikke bliver sorternet derover ? Hun er <b>Moder</b> , det er , hun har Kjcerlighed til Barnet : c . Saaledes forholder ogsaa den chrisine Kjcerlighed sig imod enhver , 1 Cor . 10 , 24. 23. Endelig er der ikke noget cedlere af alt det , som har Liv , hvorimod man kan Mennesker <b>vise</b> og bevise Kjcerlighed , end som Ncesten ; han er ikke nogen Djcevel , ikke nogen stal man Love , Bjsrn eller Ulv , itke Steen eller Trce , men han er et Creatur i alle elste frem Maader ligesom du , og der er ikke noget til i Verden , som er kjcerere , behå . ' A geligere , nyttigere , tresteligere og helligere forneden , end han er ; thi haner efter sin Art og Natur stabt til at omgaaes med andre Mennesker , stal fere sit Levnet og Forhold iblandt dem . Gud har da af alle Creature ikke kunnet

<h4>1523</h4>I ved deres Prcediken skulde faae den Helligaand ; men de sege at gjere eder ved Lcrrtil udvortes Helgene , som udvortes uden Aanden holde Heltider , visse Aars- dommen tider og Ofringer efter Loven , hvilket altsammen ere kjedelige Ting , hvoraf I have intet andet end det , at I gjere eder forgjeves Umage , og omsider faae den evige Fordemmelse . Derimod have de det , at de kan rose sig af , de ere de Galaters Lcerere , dem de have fert fra Pauli kjcetterste <b>Lcerdom</b> igjen til den hellige <b>Moder</b> , Synagogen . Ligesom Pavens Fuxsvandsere nuomsiunder rose sig af , at de igjen fere dem til Kirkens Skjod , hvilke de med deres Legn forvende . Men derimod rose vi ikke af eders Kjod , men af eders <b>Aand</b> , thi I have faaet Aanden afvor Prcediken , som forhen sagt er i det 3 Cap . 2. V . Men det vcere langt fra mig , at jeg stal rose mig , uden alene W . af vor Herres Jesu Christi Kors . Apostelen gaaer saa vidt i sin Tale , at han bliver ret heftig , og taler disse Ord som af et oprert Sind : Men det vcere langt fra mig : c . som han vilde sige : de falste Apostlers kjedelige Ros er saa stadelig og ond en Ting , at jeg vilde enste , den laa begravet i Helvedes Afgrund ; thi den gjor usigelig stor Skade . Men om nogen ikke vil lade den fare , da maa han rose sig afKjedet lcenge nok , og saa fare til Djevelen med sin forbandede Nos . Gud lade mig Christnes ikkun beholde den eneste Ros , at jeg kan rose mig af Christi Kors . Saa siger Ros . han ogsaa Rom . 5 , 3. ~ Vi rose os i Trengsel . " Og 2 Cor . 12 , 9.10 . „ Jeg vil helst rost mig af mine Skrøbeligheder , at Christi Kraft kan boe i mig . Derfor er jeg vel tilfreds i Skrsbeligheder , i Forhaanelser , ' i Nod , i Forfelgelser, i Angesier for Christi Skyld ; thi nåar jeg er strsbelig , da er jeg mcegtig." Hermed viser St . Paulus , hvad der er Christnes rette Ros , nemlig at de kan rose sig , vcere glade og frimodige i Trcengsier , Forsmcedelse og Skrebelighed : c . for Christi Skyld . 2. Verden kaster ei alene Foragt paa Chrisine , men den endogsaa hader , forfolger , fordemmer og drcrber dem som de allerstadeligsie Mennesker , der forstyrre baade verdsligt og geistligt Regimente ; thi det stal hedde , at de forsere Folk og stifte Oprer ; som og Christus selv blev bestyldt for begge Dele , at han „ forserte Folket , og forbed at give Keiseren Skat , " Luc . 23 , 2. Han maatte derfor dee som en Kjcetter og Oprerer : c . Dog efterdi Chrisine lide saadan Spot og Forhaanelse ikte fordi de ere Mordere , Horkarle , Tyve : c . men alene for Christi Skyld , hvis AZre og Velgjerninger de forkynde , da rose de sig af denne Lidelse og Christi Kors , og med Apostlerne glcede sig , at de agtes vcerdige til at forsmcedes for Christi Navns Skyld . Saaledes stal vi nuomsiunder ogsaa rose os og vcere glade , nåar Paven med sineTilhcengere forfølger , fordemmer og ynkeligen omkommer os , saasom vi lide det ikte for vore Misgjerningers Skyld , som Tyve og Mordere , men for Christi vor Frelsers Skyld , hvis Evangelium vi lcere reent og puurt . 3. Der ere da to Ting , som befceste og forege vor Ros : ferst at vi ere Chrisine visse paa , vi have den rene guddommelige <b>Lcerdom</b> . Derncest at vort Kors og Lidelse er Christi egen Lidelse . Vi have da ikke nogen Aarsag til at jamre og beklage os , nåar Verden endstjont ynkeligt forfelger os ; men vi have langt mere Aarsag til at glcrde os og vcere glade . Verden mener vel ganste vist , at vi ere ulyksalige og forbandede , thi den demmer efter udvortes Anseelse ; men Christus , som er langt heiere end Verden , for hans Skyld vi lide , han siger , at vi ere salige , og at vi stal glcede os , : c . Match . 5,11 .12 . „ Salige ere I , nåar Menneskene bespotte og forfelge eder for min Skyld , og tale allehaande ondt om eder , nåar de lyve det ; glceder og frydereder " : c . Vor Ros er da en langt anden Sag end Verdens , som ikte kan rose sig i Anfegtning, Forsmcedelse , Forfelgelse , Ded : c . men afHeihed , Rigdom , Fred , A3re , Viisdom , Retfcerdighed : c . men den Ros faaer ingen god Ende , men bliver kun til Skjcendsel og Hjertesorg .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016102848582'target=_>Glaser, Adolf, 1870, Hittebarnet</a><h4>2357</h4>Det er ikke nødvendigt for vor Fortællings <b>Formaal</b> Skridt for Skridt at folge de Eventyr , fom modte Miss Katarina fra det Tidspunkt , da hun forlod „ Det dybe Krus " og blev Kapteinens Frue ; thi stjont det vilde være saa let som jeg ved ikke hvad at godtgjore , at den unge <b>Kvinde</b> ved at folge sit Hjertes Udkaarede blot havde givet efter for en usiyldig Impuls og ved at standse hos ham en <b>vis</b> Tid bevist Dybden og Styrken af sin Hengivenhed for ham — stjont vi skulde kunne hitte paa mange rorende og veltalende Undskyldninger for begge Parters Feiltrin , saa skulde Lceseren muligens kunne faa Afsmag for saadanne Beskrivelser og saadanne Argumenter, af hvilke to Slags han desuden allerede har faaet tilstrækkeligt tillivs i den ovennævnte Novelle „ Ernst Maltravers".

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015010648110'target=_>, 1849, Almennyttigt dansk Konversations-Lexikon over alt det Videværdigste i Naturen, Kunsten og Videnskaben, fra den ældste til den nyeste Tid, i alfabetisk Orden, efter Brochhaus's Tydske Original, nyeste Udgave</a><h4>97</h4>af et afdsd Vcesen i Skikkelse af sit forrige Legeme , i Skikkelse af en Stygge . Troen paa Aanders Aabenbarelse har man kicndt fra de celdste Tider og den spiller i alle Folkeslags Mythologi en Rolle og har efter alle de Aarsager og Omstændigheder , der giver Folkestagenes religisse Meninger et forsiiclligt Prceg , formet sig paa den forsiielligste Maade . Mennesket er i Almindelighed tilbsieligt til at tcenke sig og fremstille det Guddommelige under sandselige Billeder , og hvor Foregaaelsen i hans eget Indre og <b>Kundskaben</b> til Naturkræfterne enten er usikker eller ganske mangler, der kan vel iscer levende Drsmme , do i visse Sygdomme opstaacnde Syn af Billeder, som udspringer af den Paagieldendes egen personlige Folelse , ligesom ogsaa mcegtige Naturtildragelser o . s . v. , fremkalde Troen paa en umiddelbar Indgriben af saadanne og andre Aander og paa deres sandselige Tilvcerelse og Fremtrcedelse . Ligeover for den rolige Erfaring og Videnskabernes Lys sremtrceder saadanne Forestillingsmaader hvor vigtige og interessante de end ellers kan vcere som underordnede Udviklingstrin af den religisse Tro , som Overtro , og faaer en vigtig praktisk Betydning , naar med den formeentlige <b>Kundskab</b> til Aandeverdenen forener sig en forfængelig Indbildning om Muligheden ved et Herredomme over samme at kunne frembringe overnaturlige og mirakulose Virkninger . Med Forbigaaelse af alle de forsiiellige Former , som en saadan Acmdelcrre antog i de hedenske Religioner, vil man her kun erindre om den Indflydelse, som Troen paa Dicevelen ( se dette Ord ) og de ham underordnede Aander har havt i den christne Verden . Den avlede ikke blot Magi ( se dette Ord ) , Nekromanti ( se dette Ord ) , og det hele Apparat af Middelalderens Trolddomsvæsen med dets meningslsse Vesvcerings- og Bandformularcr , men ogsaa de sircekkelge Hercprocesser , ( s . dette Ord ) ' , der underststtedes af Paverne , iscer i Tydsiland, som en Erstatning for Inkvisitionen , fandt en Underststtelse i Levningerne af den hedenske Overtro og en barbarisk Kriminaljustits, og i det 16 de og 17 de Aarhundrede stugede en Masse af Offre . Balth . Bekker ( se Samme ) , som i det 17 de Aarhundrede bekcempede dette Uvcesen , vovede dog ikke at negte Dicevelens Tilvcerelse ; men ssgte derimod meget mere at <b>vise</b> den Taabelighed , som laa deri , at anklage ulykkelige , af Naturen forssmte , syge <b>kvindelige</b> Skabninger for at staae i Forbund med Dicevelen eller andre onde Aander . Ikke blot Bekker , men ogsaa Chr . Thomasius ( se Samme ) , der i Begyndelsen af det 18 d , e Aarhuudrede angreb Hereprocessernes Uvcesen , maatte udftaae Bagvastelser

<h4>167</h4>modnere mandlige Alder . Han dsde i Buckeburg 1768. Hans blandede Vcerkcr blev efter hans Dod udgivet af Nicolai 1768 — 1781 ; andet Oplag 1790. eller Fibler er meget gamle i Tydsiland . I Danmark og Norge er Luthers den crldste . Luthers A-B- C udkom mellem Aaret 1525 3 O . Den indeholder Fadervor , Trosartiklerne og nogle Bonner . Abchasere , et transkausisi Folk under Nussifi Overhcrredemmc . Abchasicn er 6,000 Verster stort og har 52,000 Indbyggere , begromdses i Nord af Kaukasus , i Vst af Migrelien , i Syd af det sorte Hav , i Vest af Tscherkessien . Landet er meget frugtbart , frembringer Olie , Vin , Sydfrugter og Korn , men Landet er ikke vcldyrket , mange Ruiner fra Fortiden vidner om en tidligere hsiere Kultur . Abchascrne hsrer til den kraftige Tscherkesserrace og udmcerker sig som Rovere . At eie et blankt Vaaben er en Abchasers fterste Lykke . Slavehandelen med Tyrkerne var tidligere en Hovedcrhvervgren og har formindsket Folkemængden betydelig . Abdal , persisi Munk , en Begeistret . 2 lbd-el - Kader , egentlig El-Hadschi- Abd - el - Kader - Uli - Mahiddin , er tilligemed Mehemed Ali den nyere Tids betydeligste Personlighed i hele den muhamedanske Verden , af en Marabutfamilie , af Stammen Haschem , er fedt 1807 i en hans Familie tilhorende Undervisningsanstalt for Marabuter (muhamedanske Proefter ) i Noerheden af Maskara . Allerede hans Fader , der var anseet for en hellig Marabut og dede 1834 , udsvede en stor Indflydelse paa sine Landsmcend , og efterlod sin Sen denne i Arv . Hans <b>Moder</b> , Zora , som ligeledes stod i Ry for Hellighed , er en med Karakteer , <b>Aand</b> og Indsigt heist udmoerket <b>Kvinde</b> , og disse Egenskaber har hun siienket sin Sen . A . giorde i sit Bde Aar med sin Fader en Valfart til Mekka , hvorfor han forte Navn af El-Hadschi ( Pilegrimmen ) . 1827 besegle han LEgypten , hvor han i Kairo og Alerandrien ferst lcerte den Vstcrlandsie <b>Dannelse</b> at kiende . Religiest Svcermeri og Melankoli er de fremragende Troet i hans Idre . Allerede tidlig vifte han ualmindelige Aandsgaver og paa Heisiolen i Fez erhvervede han sig <b>Kundskaber</b> i den arabiske Videnskabelighed . Arabernes raa Grusomhed og lidenskabelige Kienssandselighed var ham fremmed , han vaager over Scedernes Renhed og lader sig ikke henrive af Vrede eller Lidenskabelighed . SaaledeS afskaffede han den Skik at sicere Hovedet af de droebte Fiender og indferte en mildere Behandling af de Fangne , Dedsstraffen for Wgtefiabsbrud blev forandret til en mildere Straf ; derimod har han fat Livsstraf ( Kvcel-

<h4>275</h4>den end var . dog vaadrog ham Napoleons Unaade . I den Vsicrrigsie Krig 1809 anfsrte han et Armcekorps og blev derpaa Guvernor over de Illyrisie Provindser . I Krigen mod Rusland handlede han uden al Kraft og Bestemthed , vaadrog sig endelig Napoleons hele Utilfredshed og sendtes tilbage til de Illyrifie Provindser . Aandssvag vendte han hiem til Frankrig og levede i den lille By Montbard, hvor han som Folge af et Fald fra en Havemur dede den 28 de Juli 1813. Hans celdfte Son , Napoleon , blev i Januar 1815 af Kong Ludvig den Attende bekrceftct i Titelen: Hertug af Abrantes . Han er Forfatter til en i Paris udkommet Roman : eoeilrn < ! o fomino > ( 2 ) Abrantes ( Josephine lunot , Hertuginde af ) , Nysncrvntes <b>Hustru</b> , bekiendt som Forfatterinde til vigtige Mindeskrifter , er fedt den 6 te November 1784 i Montpellier . Hun nedstammede paa modrene Side fra Komnenernes Slcegt , som , efter at have behersket det Vstromersie ( Grcrsie ) Keiscrrige , i det 17 de Aarhundrede maatte soge Giestely paa Ven Korsika . Efter sin Mands Dod levede Fru Permon , Hertugindens <b>Moder</b> , i Paris. Hun havde allerede paa Korsika vcrret en Veninde af Bonapartes Familie og forstod nu at fcengsie ham til sit Hus . Indtaget af hendes Datters <b>Aand</b> og <b>Dannelse</b> , valgte lunot hende til LEgtefcelle , cndsiiondt den forste Konsul havde tiltcenkt sin Indling et mere glimrende Parti . Hendes Mands fortrolige Omgang med Napoleon satte Hertuginden i Stand til at kunne iagttage den store Keiser i Ncerheden . Dog folte hun sig forst senere foranlediget til at giore Meddelelser af de rige Skatte , hnn havde seet og erfaret . Den indstrcenkede Stilling , hvori hendes Mands Dsd ( 1813 ) og Napoleons Fald satte hende , nodte hende til at tage Pennen i Haanden , for at kunne leve paa en nogenlunde glimrende Fod . Det forste Vcerk , hun ndgav , var : ~ HAoiNoil ' oB , ou 80 UV0lllI8 NlBlol iIZUOB BUI lll revolution , lo llirootoiro , I « cnnzmittt , l ' einniio o < l : i roBt9nrlliion . " ( 18 Bd . , Paris 1831 — 35 ; 2 det Oplag , 12 Bd . , Paris 1835 ) . En Mcengde vellykkede Karakteersiildringer og interessante Anekdoter , samt en let , underholdende Fremstillingsmaade , der dog ikke sielden grcendscr til Piat , henledte Publikums Opmcerksomhed paa dette Vcerk . Hertuginden saae heri en Opfordring til at fortscette det , og saaledes udkom : , MomoiroB Bnr IH roBt3ura < ion , ou BouveniiB ln'Btoriljuo« « nr colto la i evolution clo 1830 ot lo » ptcmioi ' t ' B Hnno6B llu clo I.ouiB- I ' ni ! ippo . " ( 6 Bd . , Paris 1836 ) . „ BoUVONII ' B 6 ' UNO HlttbaBBs < ) 6 6 t ll ' llN

<h4>1136</h4>maattet adlyde Kongens Befaling : og at han sil hende udncevnt til Grevinde af Castello Fiel . Saa Meget er vist , at hun var <b>Moder</b> til to Sonner og begav sig efter Hertuginden af Chinchons Dsd til Nom , hvor A . i Februar 1829 deklarerede sit Wgtesiab med hende . AlcudiaS Mindeskrifter udkom under Titel af : , Melnoii6B clu Prince < lo la Vaix Don Mitnuol ( I < ) l ! dv , lin « « ie L3BBHNO , eointe « i ' Nvorlnmnw , aneien pr6inier ministre < lu roi generali , < ? Bill > 6 « le « 68 amiral 6 te . < rH < luitß 6 n iranolli » < H ' apr6B . le mllnuserit pilr ( 5. <i'L!B- ( 48 d . , Paris 1836 — 38 ; paa Tydst 4 Bd . , Leipzig 1836 — 38 ) . A . ' s Broder Luis Godoy dode 1801 som Geralkaptein i Estremadura . ( 1 ) Alcudia , 1 ) Stad paa den Spanske V Majorca , har et Fyrtaarn og to Havne . Koralfisken. 1000 Indb . ; 2 ) en Flette i Spanien, i Kongeriget Valencia , ikke langt fra Alcira . Efter denne sik Fredsfyrsten Godoy Titel af Hertug af Alcndia . Aleuinus eller Alchuin ( ogsaa Albin ) , ved <b>Aand</b> , <b>Dannelse</b> og Indflydelse paa sin Tid det Bde Aarhundredes ypperste Lcerde , Carl den Stores Fortrolige , Loerer og Raadgiver , var af Angelsachsisi Herkomst og fsdt omtrent 735 i Aork i England . Han erholdt Undervisning af Erkebiskop Ekbert og dennes Slagtning Aelbert og blev , da den Sidste 766 udncrvntes til Erkebisp , hans Efterfolger som Skoleforstander i Jork . Paa Hiemreisen fra Rom , hvor han havde hentet Pallium til en Ven , lcrrte Carl den Store ham at tiende i Parma , kaldte ham 782 til sig og benyttede ham ved Foranstaltningerne til Kulturens Indforelse i sit Rige . I de Loerdes Forening ved Carls Hof forte A . Navnet Flaccius Albin« s . Carl lod ham undervise Andre ved sit Hof , i hvilket Viemed der oprettedes cn Hofskole Oelwla pnliUin » ) ; Keiseren stillede desuden under A . ' s Opsyn adskillige Klostre , paa det han i disse skulde sorge for Videnskabernes Udbredelse . De fleste Skoler i det Frankiske Rige blev dels stiftet , dels forbedret af A . Saaledes grundede han iblandt fiere ogsaa Skolen i Abbediet St . Martin i Tours 796 , hvorved han tog den i Jork til Monster , og i hvilken han selv gav Undervisning , efter at han i Aaret 801 havde taget sin Afsked fra Hoffet . Fra Tours verlede han hyppig med Carl den Store Breve , hvilke senere udbredte Lcerdommen videre i det Frankifie Monarki . Alcuin dode den 19 de Mai 804 og efterlod foruden mange theologifie Skrifter flere om Undervisning i Filosofiens , Mathematikens , Talekunstens og Sproglærens Begyndelsesgrunde ;

<h4>3174</h4>H . cm ! B < > c ' , l < 3 vns , ( L. ) , en Slutningssats . H . ( L. ) , et Forsvarspunkt . Artificiel , ( L. ) , frembragt ved Kunst ; kunstig , kunstmcessig . Artificium , ( L. ) , et ) , Kunstvcerk , Kunststykke, Kunstgreb ; Kneb , List . Artifieii . ' s , ( L. ) , kunstig , kunsifuld ; listig , snu , forslagen . Artikel , ( L. ) , en ) , 1 ) det Samme som 2 ) en <b>vis</b> Varesort (iKiobmandssproget); 3 ) ( i Grammatikken ) Kion s ord , K i endeord , angiver dels Hovedordets (Substantivets) Kion , dels om Hovedordet siaaer bestemt eller übestemt . I det Danske Sprog hedder den übestemte Artikel en ( Fcellcskion ) og et ( Intetkien ) , der scettes foran Hovedordet, f . Er . en Mand , et Hus ; den bestemte Artikel en og et , som foies bagtil Hovedordet , f . Er . Manden , Huset ; desuden har man den saakaldte Adjektivernes bestemte Artikel ( den , det , de ) , der bruges , naar der umiddelbart foran Hovedordet staner et Egensiabsord (Adjektiv), f . Er . den gode Mand , det hvide Hus , de smukke <b>Kvinder</b> ; 4 ) , 1. Artikler betyder i Sammensætninger : Indbegrebet af de for en <b>vis</b> Indretning gieldende sceregne Lovbestemmelser ; s . Er . Laugs-Artikler , Krigs- Artikler . Artikel-Brev betyder i Tosproget: Samling af Forordninger angaacnde Krigstugten , Politiet og hele Sotienesten paa en Krigsfiaade . ( 1 ) Artillerie , ( F . ) , et ) , det grove Skyts med alt Tilbehor ; det dertil horende Mandskab; Kunsten at bruge det grove Skyts rigtig , Fyrva ' rkcrkunst . Ordet Artillerie , som Nogle afleder fra det Latinske nrw < < > l ! l ' , - c > (Kunsten at siyde ) , forekommer forst hos Fransimcendene og Spanierne . Opfindelsen af det grove Skyts falder i Tidsrummet kort efter Korstogene , da Krudtet gansie havde fortrcengt Brugen af de hidtil anvendte Krigsmaskiner ; dog maa bcmcerkes , at ogsaa hine Maskiner undertiden fik Navn af Artillerie . Som nyere Vaaben er Artilleriet Resultatet af en lang Ncekke Opdagelser paa de fysikalske og mathematisic Videnskabers Gebet , hvisaarsag ogsaa dets <b>Uddannelse</b> kun eftcrhaanden kunde siride frem . For Fcrstnings- og Belcirings-Krigen hidforte Brugen af Ildvaabcn et gansie nyt System ; derfor har Fcestnings- og Beleirings- Artilleriet ogsaa udviklet sig ulige tidligere og hurtigere end Feltartilleriet , hvilket sidste derimod har frembragt en noesten endnu storre Revolution i Taktikken : man nodtes til at formere sig i Afstand , naar man ei vilde lide altfor meget af det grove Skyts , og i samme Grad som Feltartilleriet blev bevægeligere , blev Taktikken det ogsaa . Men istedetfor at forfslge den praktifie Retning , paa hvilken Krigen

<h4>3427</h4>Bedrageri opdages ved at antcende det . Den cegte Asphalt giver da en harpiragtig Lugt , brcender med en hvid Flamme og efterlader lidet Asie , som ei er Tilfceldet med det almindelige Veg , der ved Forbrændingen giver en tyk , sort Rsg . Den ucegte er ogsaa tungere og mindre brcekkclig . Kunstig Asphalt er det ved Stenkulsticerens Indlogning tilbageblivende sorte Harpir . Erfaring har imidlertid lcert , at den kunstige Asphalt , siiondt den naturlige meget lig , bliver blod i Solheden, hvilket den naturlige ikke gier . ( 1 ) eller Det asphaltiske Hav . Saaledes kaldes ogsaa Det dsde Hav , fordi dette indeholder megen Asphalt . ( 2 ) eller Asfyktisk , ( G . ) , pulslss , afmcegtig , siindsd . Asphyxvre eller Assyrere , ( G . ) , voere eller blive afmcegtig eller siindod . AsphyxZ eller Affyr i , ( G . ) , en ) , Pulsens Standsning ; dyb Besvimelse ; Skindsd . Aspirant , ( L. ) , En , som strceber eller tragter efter Noget , gisr Paastand , ssger ct Embede o . s . v. ; en Embedskandidat . Aspirzznte , et <b>Fruentimmer</b> , som ssger et Embede ; ( i Klostre ) en Provenonne . Aspiration , ( L ) , Beaandelse , Aandelyd , Aandestsd ; et Bogstavs Udtale med en saadan Aandelyd ( som om ct H stod foran Bogstavet ) ; Lcengsel , Higen ; Sicelens Ovlsftelse til Gud . Aspirere , ( L. ) , beaande ; udtale med en Aandelyd eller et H ; hige , tragte eller sircebe efter Noget . Asplund ( Arnold ) , en Svensk Lcerd , var fedt i Stockholm 1733. Han blev Student i Uvsala 1754 , Sogneprcest i Klara 1789 og Rigsdagsmand s . A . , 1809 , Fuldmcegtig i Banken og holdt 1812 sit lubilceum som Magister . Han dsde ugift den 12 te Januar 1815. Allerede fra sin tidlige Ungdom viste han en brcendende Lyst til Kundskaber, og hans hele Liv var et uafbrudt Studium. Han havde grundige <b>Kundskaber</b> ide filosofiske Videnskaber , men iscer i de theologiste. Tidligt , endog inden han forlod Skolen, var ban bekiendt for sine ypperlige Gaver til at vrceke ; og det fortcelles , at en til hin Tid meget anseet Prcest , der havde stsrre Tale- end Ovfiudelsesgaver , som oftest an- , modede A . om Dispositioner . I sin lange Tienestetid som Prcest bibeholdt han sin Anseelse som Prcrdikcmt , og Thorid , der var saa lidet tilfreds med sin Tids « åndelige Talere , kalder hans Ligtale over Nymanson . . rl-n intannin« og Skisndt han trivedes bedst iblandt sine Vsger og desuden var sygelig , viste han dog en <b>vis</b> praktisk Dygtig-

<h4>3779</h4>end tidligere dem , der bekiendte sig til det streng Lutherfie Dogma . For at sikkre dette for ny Farer , bragte han efter langvarige Underhandlinger den saakaldte Concordieformel i Stand , der , gansie imod sin Stifters <b>Aand</b> , indsnsrede den Protestantiske Lcere i en stivnet Form . Hvis Historien kunde tage Trcrkkene til A . ' s Karakteersiildring allene af de antydede Begivenheder , da maatte den lcegge mere Skygge end Lys deri og paa ingen Maade betragte ham som en af Tydsilands mest udmoerkede Fyrster i det 16 de Aarhundrede . Men her maa det ikke oversees , at han som Lovgiver , som omhyggelig Opelsier af enhver Indretning til <b>Dannelsens</b> Udbredelse , som samvittighedsfuld og sparsommelig Ordner af Statshusholdm'ngen hcevede sit Land op til et Trin , som dengang ikke noget andet i Tydsiland havde naaet . Underststtet af kloge Raadgivere , raadfsrte han sig med sine Landstcender og lagde en god Grundvold for Statsforvaltningen, hvilken alene kunde rokkes ved Misgreb af og Sorglsshed hos en Rcekke ham ulige Efterfslgere og ved ydre Storme . Statsorganismen sil ved ham en Indretning , der for den Tid var den til at giore Forretningsgangen simplere mest skikkede . I Finantsbestyrelsen adskiltes Skatterne fra Kammerindkomsterne, og hine overgaves til Stoendernes Forvaltning . Retspleien ordnedes ved en forbedret Indretning af Domstolene og ved ny Love , der under Navn af Konstitutioner indfsrte en ensformig Landsret , der ved Romerfie Normer fortrængte de gamle Tydfie Netsscedvaner. Hans Negierings mest glimrende Side var dog den Omhu , han viste for at forsge Landets Krcefter ved at ophielpe Agerdyrkning , Vindskibelighed og Handel . Idet han overalt saae med egne Vine , bereiste han sit Land i alle Retninger og lod 1566 Hiob Magdeburg gisre Udkast til et Kort over Sachsen . Der opmuntredes til at opdyrke det ode Land og til at udstykke Byernes store Godser , og Agerdyrkningen ophialpcs isoer ved msnstervcerdige Dyrkning af de fyrstelige Domcener , i hvilken Sagnet tildeler hans <b>Hustru</b> en meget betydelig Andel . I Særdeleshed stroebte han at ophielpe Frugtavlen og forte derfor paa sine Reiser bestandig Kierner hos sig for at uddele dem , firev ogsaa en „ Kunstlich Obst- und Gartenbucblein " , og bod , at ethvert ungt Par i LEgtesiabets forste Aar skulde plante to Frugttrceer . Flere Kammergodser udstykkedes og overdroges i Aruefceste , Forpagterne af Domcenerne forsynedes med Forstrifter , der var beregnet paa en neie Benyttelse af Jorden , og endog megct Hoveriarbeide afisstes med

<h4>3925</h4>ncevnte . De viser mistroifi den Fremmede tilbage og er derfor ogsaa kun lidet kiendte , ihvorvel deres Karakteer ingenlunde er ond , men meget mere synes at vcere af Naturen godmodig . Da Continentet er saa udstrakt , er det naturligt , at Australierne deles i forsiiellige Stammer , og at der i Sprog , Religion og Dannelsesgrad viser sig Forsiielligheder , der hoist paafaldende trceder stem i Folge deHimmelstrsg, de beboer ; men den ringe <b>Kundskab</b> om Landet tillader ingen Adskillelse , om man end ved mangfoldige lagttagelser ledes til at antage Vstaustralierne for de mindst vilde og utilgængelige . Religionerne er af cegte hedensk Natur , alligevel har nsiere lagttagclftr godtgjort, at de hviler paa bestemte og ordnede Forestillinger , i hvilke en god <b>Aand</b> , Koyan , og en ond , Petoyan , er fremherskende ; de ligger i Kamp med hinanden . Beboerne har nogle Forestillinger om et tilkommende Liv og et Slags Prcestesiab , der i den ostligc Del af Nyhollaud kaldes Karaji , er meget agtet , og sammes Embeder synes oftere at vcere arvelige . I vanskeligere Tilfalde giver de Raad , lceger Saar , helbreder Sygdomme og forudsiger Fremtiden. Ved Kong-Georgs- Sund ( i Sydvest ) er der ligeledes en Slags Prcester , der hedder Mulgarradock og deles i flere Klasser ; de er her hoiagtede og gicelder for Troldmcend , der kan fremkalde Regn , Uveir og Sygdomme . Aarlig holdes vaa en jcrvn , for Groes ryddet Plads en religiesFest , hvis Hovedsiemed synes at vcere at flaae Fortcenderne udaf Munden vaa Drengene , hvorved disse kommer til at nyde alle Mcendenes Forrettigheder . De Dsde dels begraver , dels brcender de , hvorved visse hellige Skikke iagttages . Nyhollcrndernes Leoemaade er omflakkende ; dog bliver de ofte i nogle Egne lcengere end ellers , naar disse frembyder sccrdeles Fodemidler . Om Morgenen gaaer de , scedvanlig to eller tre sammen , Mandfolkene vaa Jagt eller til Fiskefangst , <b>Fruentimrene</b> ud at sisie med Snorer og Kroge , at dukke ned efter Muslinger o . s . u . ; Enhver spiser noget af det , han fanger , Resten bringes hiem . Desuden forfcerdigcr Mandfolkene Vaaben og Baade , <b>Fruentimrene</b> sorger for Ovforelsen af Hytterne o . s . v. , og vaa Vandringen maa de tillige bcere det Meste , Mandfolkene bryder sig kun om Vaaben . DandS er begge Kions Andlingsfornoielse ; ligeledes fordriver man Tiden med at staaes og stentre omkring , indtil Sulten krcever ny Anstrcengelser . De delcr sig i mange smaa Stammer , der scedvanlig bestaaer af faa Familier . Hvad Nyhollcendernes Karakteer vedkommer , saa viste den sig ved Euro » pccernes forste Ankomst og viser sig endnu , hvor ikke fremmed Indflydelse har virket skadeligt ,

<h4>4286</h4>vis ved Vinhsstens Ncermelsc , og ved dennes Slutning Haloernes Fest . Efter disse fulgte LenceerncS Fest ( Vinpersefesten ) i Maaneden Chamelion , og som lun var eiendommelig for Staden Athen . Festlighederne derved , foruden Theaterforestillingerne , bestod i et stort Giestebud, hvortil Staten leverede Kiod , og i en Procession giennem Byen Athen , ved hvilken Leilighed de paa Dionysosfefterne scrdvanlige kaade Streger fandt Sted . Efter Lcn < rerne kom Anthesterierne i Maaneden Anthesierion , da den ny Vin forst blev drukket . Paa den anden Dag af denne Fest bestod Hovedboitidelighedcn i et stort , offentligt Maaltid , ved hvil « ket man , bekrandset med Foraarsblomster , under Trompeters Klang anstillede formelige Veddekampe i at drikke ; den bestod endvidere i et bemmeligt Offer for Statens Vel , bragt af Archon Basileus ' s Gemalinde , som derved paa en symbolfi Maade trolovedes med Guden . Paa den tredic Dag bragtes den chthonisie Hermes og de Afdodes Aandcr et Offer . Tilsidst fulgte de store Dionysier eller Stad-Dionysierue, son : hoitideligholdtes i Maaneden Elaphebolion, og ved hvilke ny Komedier og Tragedier kom til Opforelse . 2 ) Den triaterisie Dionysos-Fest , som aarlig midt om Vinteren afholdtes af Koner og <b>Piger</b> , saakaldte Mcenadcr, paa Vicrge til Nattetid med den vildeste Enthusiasme og under Fakkelskin . Denne my « stiste Hoitidelighcd stammede vistnok fra Thracien, og dens <b>Uddannelse</b> fores tilbage til Orpheus. Naar den fandt Indpas i Grækenland, lader sig ikke nsiagtig bestemme ; man studer den tidligst i Bootien , isoer i Theben , bvor <b>Kvinderne</b> fcirede den paa Kithceron . Et vigtigt Punkt er ogsaa Parnassus , paa hvis boiestc Top Attisie og Delphifie <b>Kvinder</b> holdt natlige Orgier til TEre for Bacchus og Apollo . Mcenaderne var ved denne Leilighed klcrdte i Stind af Hiortekalve , svang deres Thyrsus , glorde Larm med Haandpauker og dandsede med flagrende Haar paa den vildeste Maade . Ved denne ekstatiske Festligholdelse repræsenteredes Guden af det ham helligede Offerdyr , Tyren , som Mcenaderne i deres Raseri sondcrrcv . I den celdste Tid var endog Mennesieoffre ikke sieldne . Besirivelser over disse vilde Fester findes hos Digterne . 3 ) Til den senere Tids Vacchanalier lagdes Grunden i Athen lander den Peloponnesifie Krig ved Indforelscn af fremmed Gudstieneste og Conventikelucrscn . Fra Grcekcnlaud kom de til Italien . Allerede i Aaret 496 f . Chr . var den Grcrste Vacchusdvrkelsc blevet indfort i Rom tilligemed CereSdyrkelsen, og Ceres , Liber og Libera ceredes i et Mes Tempel . Til Mre for disse feiredes Liberalierne d . 17 de Marts , og dengang rigtignok

<h4>4678</h4>Hele taget maafie vceret mere skadelig end gavnlig for den cegte , sande Kunsts Fremgang . Af mere ydre end indre Natur , stener den lefterhaanden Publikums Sands for Nydelsen af det reciterende Drama , der giver mere at tcenke end at skue ; endog Operaen maa tilsidst stedse mere udarte til Balletten og de rythmisie Dandseformer, for at tilfredsstille og pirre den flovede Sands : ja man kunde vel paastaae , at Balletten paa sit ncervcerende lururisse , yppige Holdepunkt ei blot matter , men endog forringer Sædeligheden. De hyppig i Operaer indflettede Dandse fortiener siorstedels ikke Navnet Ballet , da sceduanlig neppe en Idee ligger til Grund for dem , men disses Hensigt meget mere blot er at give Dandserne Leilighed til at <b>vise</b> deres Foerdighed , samt at frembringe en behagelig Afverling . De Gamle kiendte ei den nuvcerende Form af Balletten , endsiisndt dennes fsrste Oprindelse lader sig tilbagefsre til Romernes Pantomimer , ligesom Balletter heller ikte var übekiendte i Orienten , navnlig hos Grcekerne . Balletten er vcrsentlig opstaaet til Hoffernes Tieneste og Fornsielse , naar een eller anden Begivenhed skulde fcires paa overordentlig Maade . Vi sinder Balletten i Italien i Begyndelsen af det 16 de Aarhundrede , iscer ved Turincr-Hoffet , hvor Grev Aglio befrugtede den med sit opsinderisie Gcnie , ligesom Hoffets Prindser og Prindsesser selv medvirkede ved Sang , Deklamation og Dands . Baltagerini , Musikdirektsr hos Catharina af Medici , indfsrte allerfsrft Balletten i Frankrig, hvor den snart blev saa yndet ved Hoffet , at Ludvig den Trettende dandsede med paa en af disse Balletter , hvilket Erempel Ludvig den Fiortende i sin Ungdom efterlignede . I Begyndelsen var Balletten her , svarende til Tidsalderens Retning , vcesentlig allegorisk og for det Meste smaglss . Forst mod Enden af det 17 de Aarhundrede sik den sin hoiere kunstneriste <b>Uddannelse</b> samtidig med Grundlæggelsen af den store Franske Opera . Til samme Tid optraadte forst <b>Fruentimre</b> i Balletten , medens hidtil kun Mandfolk havde dandset i samme ; dog sinder man ei fer 1790 Balletdandserinder af nogen Betydning . I Aaret 1697 blev Antoine Houdart de la Motte Ballettens Forbedrer og Reformator , i det han iscer forstærkede Handlingen og Ballet-Operaernes dramatiske Interesse . Endnu stedse forblev Opera og Ballet sammen , indtil endelig Jean Georges Noverre ss . s . ) adskilte Balletten fra Operaen, hcevede den til en egen Kunstart og tillige som tomlende Kunstner grundede en sindrig Ballet-Theori . Vincenzo G ale ott i ' s glimrende og geniale Forsog i Kisbenhavn paa at tilbagefsre Balletten i antik Betydning til det

<h4>5770</h4>Holland til London og senere over Frankrig til Italien . Han kom tilbage om Sommeren 1700. B . eiede grundige <b>Kundskaber</b> , navnlig i de celdre og nyere Sprog . 1699 udncevnteS han til Professor i Latinen i Upsala og vandt i denne Stilling stor Agtelse . Arbeidsom og begierlig efter at samle <b>Kundskaber</b> , sparede han for lidet paa sit Helbred ; ved mindre Sygdomsanfald enten negtede han sig al Hvile eller indstrcenkede den saa meget som muligt . Med sin anden <b>Hustru</b> , Katharina Daurer , som han inderligt elskede , og hvis tidlige Bortgang ( 1709 ) lagde ham i Graven nogle faa Uger efter , havde han tre Bern , af hvilke det yngste » ar Sennen Johan Arndt 8. , fedt i Upsala 1707 ; han blev Sekretcer i Hofkancelliet og sit Afsked med Lagmans Bcerdighed ; ded 1765 ; hans <b>Hustru</b> var Katharina Hermonia, Datter af en Prceft i Stokholm . Hans Sen var den navnkundige Vellman ( Carl Michael ) , en af Sverrigs og Nordens herligste Digtere . Han blev fedt i Stokholm den 4 de Febr . 1740. Sin fsrfte Undervisning fik han i Marice Skole , og Haand i Haand med den gik en saare omhyggelig Opdragelse hiemme i Forcrldrenes Hus , hvor fiere af Datidens mest dannede Mcend jcevnlig samledes . Stsrft Indflydelse synes dog i hans Barndom hans <b>Moder</b> at have udevet paa ham , thi hendes Sands for Musik og hendes poetiske Gemyt knyttede hende neie til Sennen , der igien elskede hende med inderlig Kicerlighed . Da han var bestemt til Studeringer , fik han Undervisning i de celdre og nyere Sprog af fiere Lcerere , deriblandt een , om hvem han veed at fortcelle , at han felv engang i Forening med een af sine Kammerater har givet ham Prygl . Snart viste det sig , at B . ' s Modbydelighed for den kolde Forstands Abstraktioner ikke havde sin Grund i Dovenskab eller « andelig Sleuhed , men deri , at han af Forsynet havde modtaget et heiere Kald . Engang i sine Drengeaar faldt han nemlig i en haard Sygdom og begyndte da under Feberen at improvisere VerS for sin <b>Moder</b> , saa at alle Tilftedevcerende faldt i den fterfte Forundring . Hans fforceldre gav ham derfor en ny Lcerer i Magister Klas Ennes , som veiledede ham i LEfthetik og Poesi , og hvem han siden sil Leilighed til i Gierningen at <b>vise</b> sin Taknemmelighed . Han forskaffede nemlig sin gamle fattige Lcerer et Prceftekald ved sin Forben hos Statssekretcer Schrsderheim. B . optraadte allerede i sit 17 de Aar som Forfatter , men der fattedes meget i , at hans ferste Forseg gav nogen Formodning om de Vcerker , han senere stulde frembringe . HanS forste Skrifter var Oversættelser , nemlig :

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010061506031'target=_>Funcke, Otto, 1880, O. Funcke's Husandagtsbog</a><h4>193</h4>ingen Profeter ere og ikke kunne skue ind i et Menneskes inderste Natur , men forst kunne forståa den af dens Udviklina , saa vcelge vi vore Bsrns Navne efter allehaande ydre og tilfceldige Hensyn . Naar imidlertid Esaias , ovlyst as Guds <b>Aand</b> , ncevner oen tilkommende Messias og Fredskonges Navne , saa vide vi paa Forhacmd , at Frelserens inderste Vcesen maa verre tilkjendegivet med disse Navne . Messias ' s Navne og Natur maa svare til hverandre . Den , som kan lcese hans Navne ngtigt , kjender ogsaa hans Hjerte , hans Hensiater , hans Vilje , hans Regjering , hans Gjerning . Nu , hvad heder da det  Barn " , som er udkaaret og fodt til vor Fredsfyrste og saliggjørende Verdensherster ?  Han skal kalhes « Jesus " , sagde i sin Nd Engelen Gabriel ( Luk . 1 ) , og MMa , Frelserens <b>Moder</b> , tillaade ogsaa sit Barn formedelst Troen dette stolte Navn . Vi sige : dette stolte Navn , thi vi vide , at dette Navn er over alle Navne . ' Hvad kunde man vel forestille sig af Lcenasel og Haab , som ikke er udtrykt i det Navn Jesus sFrelser ) ? Men for Profeten Esaias er dette Navn endnu ikke blevet klart . Med Betegnelserne Raadaiver , vceldig Gud , Evigheds Fader , Fredsfyrste ere vistnok de mest trssterige Sider af lesusnavnet fremhceveoe . Men , som det fsrste Navn ncevner Profeten et andet . Og dette vilde alle Tiders Filosofer og <b>Vise</b> ikke vcere komne paa . Dg dog er det vigtigst at vide dette , naar man vil forståa Frelseren og ikke forarge sig paa ham .  Nnder " , saaledes lyder det fsrste Navn , og dette fsrste behersker alle de fslgende . Thi  et Under " vil Frelseren blive som Raadgiver ,  et Under " i sin Vcelde ,  et Under " som Helt og Fredsfyrste . Vi undre os ikke mere over , at Frelseren kaldes  Under " . Vi ere vante dertil . Allerede som Born have vi under luletrceet fremsagt dette skjsnne Bibelsprog . Men egentlig Mde man doa synes , at den , der var bestemt for Alle , ogsaa maatte vcere ret klar og forstaaelig for Alle , og ikke et Under . Hvad vil da dette sige ?  Undere paa Jorden ere Natur i Himlen " , har en <b>vis</b> Mand sagt . Dermed er da vistnok bevidnet , at „ Underet " ikke er Noget , som i og for fig og i sig felv skulde vcere uforstaaeligt eller vel endog fornuftstndigt . Saaledes tale vi jo ogsaa om underftlld Skjsnheo og om underfuld <b>Visdom</b> , forat udtrykke Skjonhedens og <b>Visdommens</b> hsjeste Ros . Men det ligger der i Navnet « Under " , at for os Mennestebsrn , som endnu vandre i Dunkelheden , hvis Erkjenoelse endnu er uendelig indskrcenket , hvis Ojne ere tilstorede , hvis Sv , nskreds er begrcendset , at for os vil og maa Frelserens Vcesen , Virken og Liden vcere hemmelighedsfuld . Naar forst Dcekket er taget fra vore Vjne , naar vi forst faa stue ind i Dybet af Menneskets Vcesen og i Dybet af Guddommen , — naar fsrst Guds ustabte Lys ganste faar gjennemstraale os , — da ville vi ikke lcenaere vide af noget Under , da ville vi ikke mere kalde Frelseren  Under " . Men nu , saalcenge det endnu mere eller

<h4>221</h4>at ogsaa denne frygtede , nemlia for Dmvelen . Saa gik da Kristopher i Djcevelens Tjeneste . Men enonu var han ikke ved sine Vandringers <b>Maal</b> . Da nemlia Djoevelen en Dag gjorde en lang Omvej , forat komme forbi et Kors , og han ikke kunde ncrgte , at han frygtede den Korsfcestede , besluttede Kristopher fra denne Stund af at tjene Kristus . Men han var ikke saa let at finde og at gribe . Af en from Eneboer blev Kjcempen endelig belcert , at det vilde vcere Kristus velbehageligt , naar han bar Pilegrimene frem og tilbage over en <b>vis</b> , rwende Strsm , og saaledes bestyttede dem fra at drukne . Kristopher blev mange Aar tro og taalmodig i denne tunge Bestilling, dog begjcerede han inderligt nu endelig ogsaa engang at faa se sin Herre Kristus selv . Da lsd der i en msrk Stormnat en blod , klar Stemme over Stwmmen :  Kristopher , scet over " ! Manden stod taalmodig op og vadede gjennem Strommen . En liden venlig Vreng modtager ham her med Spsrgsmaalet : om han vel tiltroede sig at kunne boere ham gjennem Våndet ? Smilende og ligesom legende hcevede Kjcempen Barnet op paa sine mcegtige Skuldre og begyndte sin Fart . Men , hvad var oet ? Tungere og tungere blev det lille Barn , alt medens han bar ! Kjcempen bcevede gjennem Mary og Ben , Sveden stod ham i Panden , og havde ikke hans Stolthet ) forbudt det , kunde ban have skreget hsjt af Angst . Med Msje naaede han den anden Bred og satte Drengen ned paa Jorden . Men et nyt Nnder steede . Barnet forvandlede sig til en twj , hellig , lys Skikkelse , fuld af Storhed og Majestcet ; og se Jordkloden laa i hans Haand . Da anede Kristopher, at han havde baaret sin Herre selv , og tilbedende sank han ned og sagde :  Dia vil jeg tjene og hjcewe , og du syntes mig kun let . Men du er bleven nng for tung . Saa beder jeg dig da saa inderligt : Hjcelp du mig , saa er jeg hjulpen for alle Tider ! " la , saaledes gik oet ikke blot med Kristopher ! Vi alle , naar vi ville tjene Kristus , synke i Floderne , hvis ikke han tiener os . Thi dybe og forunderlige ere hans Stier , og hans Vei ssrer gjennem store Vande og kun ved ham blive vi hos ham . Net ville vi idaa lcere at forståa , da vi igaar have skuet den frydefulde Glans as Kristi Skikkelse , som vil oplyse , gjennemstraale og saliagjsre alle Skabninger . Vi finde idag Jesu Forceldre med deres lille Barn i Temvlet , forat fremstille det for Herren efter Loven . Store , fremtidsrige Ord har den gamle Simeon proseterende talt om Barnet . Men derefter fceftedeHan sit Blik paa Moderen  og velsignede hende " ( o , underfulde Velsignelse ! ) :  Ogsaa din Sjcrl stal et Svlrrd gjennemtrlrnae . " Hvorledes maa ikke den unge <b>Moder</b> have faret sammen i Angst ved disse den cervcerdiae Gamles ncesten grufuldt klingende Ord ! Indtil da havde hun af Engle og Mennester kun hsrt ssde , hilsende , frelseforjcettmde Rsster . Hun var bleven priset salia og havde priset sig lelv salig , fordi hun stulde blive Messias ' s <b>Moder</b> .

<h4>276</h4>sige ; —  dn er min Gud , " saa er Alt givet dig for Tid og Eviahed, som dutiltrcenger : — den smmeste , mest medfolende Kjcerligheo , oen sikreste Tilftugt i enhver Nsd , oen fuldkomneste <b>Visdom</b> , som ikke mangler i noget Opdraaelsesmiddel , Almaaten , for hvilken der ikke sindes Hindringer eller Vanskeligheter .  Dn er min Gud " , saa < ledes tsr du sige . Det vilde vcere uforskammet at tale saaleoes , naar ikke Gud selv havde lagt dia det paa Lceberne . Vil du nu virkelig verre Guds , saa tsr ou kalde ham din egen , som tilhsrte han dm alene . Visselig , der er Millioner i alle yimmelstrsg , som gjsre Fordring paa ham . Men ligesom de utalliae Blomster , som vente paa Solen , derfor ikke modtage mindre fra oen , fordi om de ere mange , men enhver modtager asSolen Saameaet , som den blot kan optage og benytte , saaledes er det oasaa mellem Gud og dig . Hvor forvovent det end lyder , du tsr anse ham , den Almceatige i det Hsje , som din , som er helt for dig og gjennemfsrer hertigt Alt med dig . Dermed har Gudsbarnet vistnok ingenlunde nogen Sikkerhet ) for , at hans Livsdage stulle vcere glattere og lettere end anore Mennesters . Men naar dit Skibsanker hviler paa Evi ^ hedsgrunden . naar Guds Vilje er den Vind , som fylder dine Sesi , kans Ord er dit Kompas og Jesus Kristus dit Skibs Styrmand , da er du <b>vis</b> paa , at du stal komme til den bedste Davn . Lad end mangfoldige Klipper og Sandbanker true dia , lao end de vilde Bslger gaa frygteligt over Dcekket — gaa under vil dit Skit » ikke . Lad oine naturlige Krcefter ovstides , dit Ansigt fures , dit Håar graanes , — Gudsbarnet er som oet Trce , der vel dsr nedentil , men grsnnes og vokser saa meget frodigere i Toppen , ja som strceber saa meget kraftigere i Hsjden , jo mere det uddsr nedentil . O . at vi nu engang gjorde fuldt Akior af det med dette :  Min Gud er du " ! Men oerpaa mangler det desvcerre mest . Vi have beklageligvis aldrig nogen ret Nd for Gud , og derfor have vi heller Intet fra ham , mgen Kraft , intet Lys , ingen Tugt , ingen Trost . Naar vi blot hver Dag aanste vilde vie ham en eneste halv Time , og for . hans Ansigt msre os det klart , hvad det betyder:  Min Gud er du , jeg er dlt Barn " , — hvor glade vilde yl da ikke sige : , Mine Tider , — med Alt , hvad de bringe af Sorg og Glcede , Tab og Vinding , Smerte og Fred , mine Tider staa i din Haand . " Da vilde vi vcere mere tilfredse end den Mand , som ikke forstod at forvalte sine Kapitaler og derfor lagde dem i en anden Mands Hcender , om hvem han vidste , at han forbandt den hojeste Kjcerlia.hed og Uegennytte med den hojeste Dygtighed og Kloaskab . Yl kunne da her i Tiden enfoldigt og Pligttro Dag for Dag udrette , hvad der foreligger os , og saa ganske roligt overlade til ham , hvorledes han uowlder Tiden , og hvad han sender os i Tiden . Vi tsr rolig lade alle VEngstelser for Arbezde , Sundhed, <b>Hustru</b> og Bsrn , for det Idre og det Indre fare . Hvorfor

<h4>350</h4>det rigtiat oa fast kan scette det ene lille Ben foran det andet , indtil der af Lyden er blevet et Ord , af Ordet en Scetning o . f . v. Alle den legemlige Udviklings Trin har Jesus maattet gjennemgaa og ikke oversprinae noget . Og ikke anderledes var det i hans aandelige, nej hellerikke i hans religwse Liv , kun at Alt skred frem uden more vH ydre Fejl , under den guddommelige Naades og den helligede Moderkjcerligheds Solskin . Guds Naade var med ham paa en helt ejendommelig og enestaaende Art og Maade . Ogsaa Jesusbarnet havde at kjcrmpe med Synden ; men han har aldrig lagt under for den . Som hos intet andet Menneske virkede paa ham himmelske Aands- og Livskrefter , og for disses lukkede han mere og mere sit Hjerte op. Med Forundring maa Maria have feet det , men visselig har hun ikke , end ikke med en Antydning , forraadt for Barnet dets Fodsels Hemmelighed . Dermed havde hun jo aldeles forstyrret al barnlig Umiddelbarhed . Jesus vidste ikke Andet , end at Josef var hans Fader , selv til den tolvaarsgamle Drena i Templet sagde Moderen :  din Fader " og jeg have lcdt ester oig med Smerte . Maria har i kydsk <b>Visdom</b> overladt det til Guds <b>Aand</b> at lsfte det hemmelige Slsr for ham . Forst efterhaanden lcerte Jesus sig selv at kjende , indenfra belcert ved guddommelige Vidnesbyrd . — Nacrr der Intet videre berettes os om Kristi tolv forste Leveaar , saa vidner just denne Taushed tydelig nok om , at , hellerikke noget Scrreaent , t Ojne Faldende , er skeet . At Barnet ikke har gjort undere, som visse Fabelboger fortcelle , derpaa behsver man sor forstandige Mennesker ikke at spilde Ord . Som Bsrn glcede sig , saaledes alcedede ogsaa han sig , ligesom de vokse , undervises , udfsre smaa Arbejder , ' hjcelpe til , lege , gjsre Tjenester , saaledes var det ogsaa med Jesus . Paa sin omme Moders Kncr ' lcerte Barnet at tale de fsrste Ord , i henoes sjcelfulde Ojne saa han med Henrykkelse mere end jordisk Glcede og jordisk Fred , af hendes hulde Lceber lcerte han fsrst at bede med Psalmernes Ord , og begeistret nedlagde hun i hans opmcerksomme Sjcel den hellige Historie , hvorledes Jehova havde talt og handlet med sit Folk i de gamle Tider , og hvad han havde forjcettet Israel i Oftfyldelsens Tid . O , hvilken Lyst det maatte vcere at undervise et saadant Barn i Herrens Veje . Der gives jo overhovedet for en Troende neppe en skjsnnere Gjernina, neppe en Gjerning , som bringer mere Glcede og Opnmntring , Tillokkelse og salig Beskjcemmelse med sig end denne at plante det hinunelske Ord i Barnesjcele . Men nu tilmed i et Barns Sjcel , der er frit for al Vanart ! Ja , sandelig , naar man dristig kan paastaa , at aldria havde et Barn paa Jorden en saadan <b>Moder</b> , saa kan man paa ben anden Side med endnu storre Ret sige : Aldrig havde en <b>Moder</b> et saadant Barn . Aldrig var der et Barn , som udviklede sia . saa harmonisk, som paa ethvert Trin af fin Udvikling udfoldeoe sig saa vidunderligt ; aldrig var der et Barn med saa hsje , cedle Gaver ,

<h4>791</h4>ved <b>Kvindens</b> symvatbetiske Kjcerligshandling i Bethanien . Hun reddede paa en <b>vis</b> Maaoe den mennestelige Slcegts Mre . Begivenheden henscetter os under det fredelige Bethanias Palmer og Vinstokke , denne Landsby , som saa ofte tjente vor Frelser til en Hvilens Havn efter den fanatiske Stads Kampliv , Storm og Larm . Her ftnoe vi et saa forunderligt Bordselskab , som det vel aldrig har vcrret og vel aldrig vil blive vaa Jorden . ( Smlgn . Joh . 12 , 1 f . ) Den , for hvis Skyld Alle ere der , er Ichls , hele Verdens Frelser , al Dsds Dso , al Livets Kilde og <b>Maal</b> . Derncest er der de tolv Apoftle , de Moend , der ere kaldte til at bcere Kristi hellige Ild ud til alle Lande , og stille Verden Jesu Billede saaledes for Oje , at det maa blive uforglemmeligt ial Evighed . Men ak , mellem dem er En , Judas , der , kaldet til det Hsjeste , formedelst Urenhed er nedsunket i det msrkeste Dyb ; — En af de — forbaabentlia — Faa , om hvem selv den evige Kjcerligheds Muno taler :  Bedre , at han aldrig havde vceret fsdt " . Vi se videre Vcerten i Huset, Simon , som veo Jesu Undermagt er bleven lsst fra Spedalskhedens forfcerdelige Plage : ja , der er endog et Menneske , der allerede engang har vceret ood og nu igjensioder mellem sine Venner ved Bordet , Lazarus . — Hans Soster Martha er lykkelig ved i et saadant Selsiab at turde opvarte ved Bordet , og hvor gestjceftia hun end er , kan hun doa ikke gjsre nok . Den Stilleste og mindst Isjnefaldende i dm hele Kreds , hun , der vel ogsaa i deres Ojne var den Ringeste , nemlig Maria , skulde snart imod sin Vilje buve Hovedpersonen ved Siden af Jesus . Indtil nu havde hun , stille iagttagende , lyttet til Jesu Ord , og med Blikke af den inderligste , dybeste Kjcerkghed nedscenket sig i ham . Nu reiser hun sig ; uden at sige et Ord tåger hun en kostbar Alabastkrukke , hvori der er den kosteligste Nardussalve , som vel det hele Asterland forstaar at skaffe ; hun ncermer sig Jesus med den og fremdeles uoen med et Ord at antyde , hvad hun gjor , gyder hun med kongelig Rundhaandethed det hele Indhold ud over Jesu Hoved . Endnu ssnderbryder hun Krukken , forat ikke en eneste Draabe stal blive igjen derinde . Duftende udgyder sig den kostbare Salve over Jesu Legeme , og underfuld Vellugt fylder det hele Hus . O , vel dig , du huldsalige Kvlnde , Duften af din hellige Kj<rrliahed er det Skjønneste i denne Nardussalve ; du bringer mg selv iom Gave . Derfor er den ikke , — som den gjerriae Judas udregner — 60 Daler voerd , nej , alle Jordens Milliaroer kunne ikke ovveie den . Vi ville her ikke tyde <b>Kvindens</b> Kjcrrlighedsvcerk , det vtl Jesus selv straks gjsre , — kun saameget sige vi allerede her : Hun udtaler gjennem sin Gave sit hele Hjerte ; hun kan ikke andet , hun maa vaa en eller anden Maade bevise Herren sin dybe , inderlige Kjcerlighed ; hendes Hjerte vilde ellers sprcenges . Dag og Nat har hun vel grundet vaa , hvorledes hun skulde kunne gjsre dette , saa har hun endelig fundet denne Vej . Derved at hun netop anvendte

<h4>974</h4>en <b>vis</b> Erstatning . Han kjender disse cedle rene Sjceles dybeste Trang . Han har forstaaet , at netop disse To have mest tilfcelles , at de have den dybeste Forstaaelse af hinanden , at de med deres hele Folen og Tcenken , Gjsren og Laoen , Elsken og Liden bedst ville lunne supplere hinanden . At Jesus derved tillige vilde befri sin <b>Moder</b> fra alle Nwrinyssorger , er visselig ogsaa rigtigt . Kun saadanne Folk , der ikke hende til , hvor smertelig Sorgen for det daglige Brsd kan vcere , kan sinde dette uvcerdigt eller vel endog latterligt . Men Herrens Hensiat gaar dog meget dybere .  Fra den Time tog Discipelen hende til sig " , — med dlsse Ord betegnes en Forbindelse , hvori al Omhed og . al Rigdom af Kjcerlighed mellem <b>Moder</b> og Son , Discipel og Discipelinoe , Ven og Veninde vare forenede . la , den Forbindelse , som Jesus stifter , og som opblomstrer under frelserens Kors , den rcekker langt ud over alle blot naturlige Familje- og Venskabsbaand . Den Forbindelse , hvori Jesus er den egentlige Sjcel og den Tredie i Forbundet , er hcevet over alle Tidens Storme , og om endogsaa Verden forgik , maatte den forblive urokket . Fsrst da er der i et cegteskabeligt eller venstabeligt Forhold den sande Forming , naar begge Parter have en hsjeste Lomgsel : at Jesu hellige Kjcerlighed alt mere og mere maa gjennemtrcenge , oplyse og helliggjme dem ; hvor begge staa paa den samme Forbarmelsens Grundvold , begge i den samme aandelige Tugt bsje sig for den samme Herre , have det samme <b>Maal</b> for Oje . Hvor Mennesker, der i Sanohed ere saaledes sindede , finde hverandre , ville de hurtig fatte en forunderlia Tillid til hverandre ; den Ene vil ikke frygte for at lade den Anden faa et Indblik i al hans Glcede og Smerte , ia , selv i hans Anfcegtelser , Svagheder oa Kampe ; — de ville let ssaa Broer over de Forskjelligheder , som beståa mellem dem , da der dog i Hovedsagen bersker Enhed ; de ville vcere fjernt fra alt Smigren , men glade i den virksomme Kjcerlighed , og ogsaa deri vil Kjcerlmheden <b>vise</b> sig , at den Ene foreholder den Anden hans ukjendte Feil og Vildfarelser ; thi evig Fuldkommenbed har jo Jesu timelige Skole og ogsaa hans Difciples Samfund t : l <b>Maal</b> . Kjender du saadant Venstab og Samfund ? Er det saaledes mellem dig og din VEgtefcelle , mellem dig og dine Venner ? — Ak , hvor faa g. rundcerlige og derfor oasaa velsignelsesrige Forbindelser gives der i Verden ! Og en sand , Md Forening er dog et saa cegte mennesteligt Behov ! Kun i Hengivelsen sinder man sia selv . Hvorfor er da den sande Forening saa sjelden ! Ikke sandt , fordi Kristi Sandhed og Kjcerlighed er saa sjelden ? O , alle I , som klage over , at I savne den inderlige Forening med Andre , stiller Eder trsstige med Johannes og Mana under Korset ! Manden , som hang . der , kjender Eders Trang i dens dybeste Grund . Skuer kun op til ham og beder til ham ! Havde han Maat til midt i sin Dod at knytte evig faste Forbund , er hans Magt Me ringere idag .

<h4>1205</h4>lader Sine hungre ! Som om i det Hele noget Andet end Synden kunde vancere Mennesket ! Dernoest er der ogsaa Mange , som stamme sig over dette og hint hos sine Medmennesker . Man mener , det stader Ens Anseelse , Ens Stilling i Samfundet at tale fortroligt med denne eller hin Mand eller <b>Kvinde</b> , at gaa med dem paa Gaden eller at blwe staaende hos dem . Nei , det gaar ikke , om end Hjertet oa Medlidenheden driver dertil . At tale et fortroligt Ord med Tjenerne , som dog skulde vcere os kjcere Kusfceller , det kalder mangfoldige Familjer for uopdraaent , for neovcerdigende , for stadeligt for Respekten. Hvilken indbildt TEresfslelse , hvilken hul Fornemhed ! Vi ville imod dette kun gjsre dette ene Sporasmaal : Har da Jesus nogensinde skammet sig over den ringeste Betler eller den dybest sunkne Synder ? Har han ikke behandlet dem alle som Brsdre og Ssstre ? Og Snster ikke du — der dog staar saa langt dybere under ham end det usleste af dine Medmennesker under dig , — at han uden Vcemmelse stal trykke dia til sit Hjerte ? Altsaa — men sandelig denne Del af Kristi Efterfsmelse vasser ikke for os ! Ak , hvor forkjerte ere oe fleste Menneskers Meninger paa dette Omraade , og hvor blive ogsaa for Størstedelen Bsrnenes Anskuelser forvrcengte derved , at de oplcrres til at bedsmme Tingene , Arbejdet , Menneskene og Forholdene i Verden efter sit ydre Prceg og Udseende istedetfor at se paa deres indre Vcesen : — forvrcengte derved , at man lcerer dem at lcegge Vcegten paa , hvad der dog kun er Klasse- og Standsforstjelle , annindelia Omgangstone eller Skik oa Brua , — paa en <b>Dannelse</b> , der mer er Misdannelse og Forfcengeligheo end sandt Hjerteliv ! Maatte vi nu af vor Herre og Frelser lcere ikke at stamme os over noget Menneske , kun at stamme os over Synden og fremfor Alt at stamme os over vor egen daarlige Hofflrrdighed , ikke at skamme os over noget Arbejde eller noget GMemaal , iscer naar vi derved kunne <b>vise</b> Kjcrrlighed . Alt , hvad der flyder af Kjcerligheden og fowger Kjcerligheden paa Jorden , det er opbyggeligt , vet er guddiMmeligt stort , det er Frelseren vcerdigt .

<h4>1637</h4>fra Guds Rige . Apostelen bevidner jo her . at de Born , der iog for sig ere <b>aandelig</b> urene , helliges veo Foreldrenes Tro , og at der mellem VEgtefeller , der jo ilke blot ere legemlig , men ogsaa sjelelig sorbundne , sinder Sted en lignende helligende Indvirming af den troende Part paa den vantro . Nu maa vi vistnok vogte os for at stutte Formeget af dette , ellers kunde der let fremvokse et farligt Letsind heraf . At der ikte kan vere Tale om ad naturlig Vej at fremtvinge et nyt aandeligt Liv , og at hellerikke det ene Menneske saaledes uden videre kan trekke det andet med sig til Himmelen , forstaar sig af sig selv . En <b>Hustru</b> kan ikte blive salig , fordi hendes Mand er en Knsti Discipel , dersom hun ikke selv kommer til Troen paa Frelseren . Bornene finde ikke det himmelske Fedreland blot derved , at Foreldrene vandre paa den trange Vej . Den yngste Broder undgaar ikke Dommen , sordi den eldste har domt sig selv og faaet sine Synders Forladelse . Per heder det : . . Enhver har sin Vej og Enhver sit <b>Maal</b> . " Og ikte siger Frelseren : Den , som tilhsrer en from Familje , stal blive salig , men : « Uden at Nogen ( du , som leser dette ) bliver fodt paany , kan han ikke komme ind i Guds Rige . " Alt dette er klart . Men et andet Sporgsmaal er det , om ikte derved , at til Eks . en Husfader har optaget Kristi Liv i sig . — om der ikke derved udgaar en stille , megtta , om end langsomt virkende , Indflydelse ogsaa paa de ovriae Familjelemmer , — en Indflydelse , der ogsaa gjsr dem , om de ikke Ugefrem kjempe imod , i sin Tid modtagetige for Troen . Har du betragtet det Under , som foregaar , naar man paa et vildt Rosentre indpoder en Gren af et foredlet ? Er ikke det et stort Under , at den indpodede klle Gren snart har forvandlet den hele store Stamme , at alle den vilde Stammes Safter og Krefter mart blive dragne med ind i den indpodede Stammes Saft og Kraft ? Er det ikke underfuldt , at den hele Stamme nu snart ikke mere som forhen berer vilde , usknmne , ikkeduftende Blomster , men kun foredlede Roser , yndige i Farve , Form og Duft ? Men dette , er et Billede af det indre Liv . Du , der som et enkelt Lem af en Familje virkelig har fundet de evige Kilder , du kan ikke let overvurdere din Post som Idmyghedens og Troens Salt . Men du kan hellerikke vere varsom og omhyggelig nok i at vandre varlig og bere dit Hjerte i din Haand , saa at Dme forståa det , at der virkelig er kommet et nyt Liv , en ny Kjerlighed , en hsjere Glede og . Haab i dig . Kun at du stadig foreholder dig selv , at du just i dm Familje maa vere saare sparsom med Ord og Tale og desto mere tenke paa Forbsn . Mdmyghed og Kjerlighed . Men da stal du ogsaa vere ved godt Mod . Tro det trsstig , den nye <b>Aand</b> i dia vil gribe de Andre i det Stille , din Tro vil ogsaa have salige Folger for din Familje , selv om du for det Fsrste ikke faar se Noget af disse Fslger , men kun maa hsre idel Modsigelse .

<h4>1836</h4>Liv tjenende Kristus . Af hellig Frygt for Kristus , — af den Fryat , som maa udspringe af Tro oa Hcerlighed , — skulle vi frwiuig verre hverandre underdanige . H samme Grad , som vi her paa Jorden blive den tjenende Kristus lige , i samme Grad skulle vi dcroppe blive den forklarede regjerende Kristus lige . Dette klinger igjennem alle Kristi og Apostlenes Ord , og skulde det ikke vcekke en hellig Frygt for Kristus i os ? Ind i en Verden , hvor den Ene helst vil bruge den Anden til sin egen Nytte , raaber Apostelen dette Ord :  Vcerer hverandre underdanige ! tzener hverandre , hvor og hvorledes og naar I kunne hjcelpe og tjene . " Ordet angaar Alle , som kalde lews Herre , ligemeget om han er Kejser eller Slave , Mand eller <b>Kvinde</b> , Filosof eller Daaarbejder , Feltmarchal eller Trcensoldat . Enhver stal vide og erkjende sig som den Andens Tjener , med hvem Gud har sat ham sammen . Ikke at Apostelen vil udstette Stand og Standsforskjelle . Den , som kjender det nye Testamente , han ved , hvorledes Apostlene formane til at give den Wre , som tilkommer VEre , at aive den Told , som tilkommer Told , at <b>vise</b> den VErefrygt , som tilkommer LErefryat , og at vcere Nvrigheden , som Guds Tjener , underdanige . Han veo , hvorledes Apostelen krcever hdmyg Underordning af den tjenende Klasse , hvorledes han klargjor og sastscetter ben dybe Forstjel mellem Mand og <b>Kvinde</b> , Foroeldre og Bsrn . Nej , ikke ar oplssc eller at bringe de forssjellige Steender i Oprsr er Apostelens- Hensigt , men han vil bringe en ny <b>Aand</b> ind i dem , Forsonlighedens og Kjcerlighedens <b>Aand</b> , den gjensidige Agtelses <b>Aand</b> og Lcengselen efter at lykkeliggjsre hverandre . Alle , saavel Hoie som Have , skulle i Gjerning og Sandhed betra gte hverandre som Brsdre , som Lemmer af et Legeme , som Born i et Hus , som Pilegrime mod det samme <b>Maal</b> , som delagtige i det samme Haab , som Arvinger til en og den samme Salighed og Herlighed . Og hvor der gjores Alvor dermed , skulde det der ikke vcere let at tjene , ' skulde det ikke der vcere en Lyst at hjcelpe , skulde det der ikke vcere selvforstaaeligt , , at man tcenkte sig ind i oen Andens Stilling og forstede efter , hvad der mangler ham , hvad der nytter oa gavner ham ? Den , som over fig saaledes i den Kunst at tjene , han ved ogsaa , at just denne Azenen gjor Sjcelen fri for Trcellefryat og Slavesind ; thi den er et yrefteligt Kald , et guddommeligt Vcert . Ingen Videnskab og Kunst gM Sjcelen saa gudlig som det at tjene i Kristi Skole . Heraf fsdes det « edleste Friyedssind , der da ogsaa , om saa maa vcere . rolig og trsstig kan modstaa en rasende Verden , naar den krcever Uretfcerdighed og Forncegtelse af Troen . Derfor har den sande Kristendom havt saa mange Vidner paa Baalet og Skafottet . Altsaa , ikke paa Sammenblanding og Ophcevelse af Standsforskjellen har Apostelen tcenkt , hellerikke paa at tilintetgjøre den personlige i dette Ords rette Forstand , men paa at tilintetgjsre

<h4>1880</h4>barnlige nok . Vi forståa ogsaa , hvor viatigt det er , naar vi ere blevne lunkne og styve og trcege oa. jordiske og uaarvaagne i vori indre Liv , eller naar vi have forvlldet os ind paa denne eller hin onde Vej , — vi forståa , hvor vigtigt det er , at Herren da wgter og vcekker os , forat vi atter kunne komme til os selv og til ham . Ogsaa vi have ofte kunnet bede med David :  leg takler dig , at du ydmyger mg ; thi naar du ydmyger mig . gjor du mig stor . " Ja . alt dette fatte vi . Men takke for alle Ting ? Ogsaa for det Onde ? Vi ville slet ikke tale om Tandpine , og alslags smaa og store legemlige Lidelser , om kaldsmcessige Forlegenheder , regnsulde Fodrezser, m . M . I denne Henseende faar man ofte straks efter de legemlige Lidelser Trost og Lys , idet man faar se den guddommelige Klcerlighedshaand , der vrrker bag dem . — Men hvor megen dyb Smerte gives der ikke i Verden ! Den Ene sorger over Tabet as en trofast , cedel <b>Hustru</b> , den Anden over et haabefuldt Barn ; denne Kustru sorger over det ulykkelige Wgteskab , hvori hun lever , — hine Forceldre ere blevne graa for Tiden paa Grund af en forloren Son . Hvorledes kan man da takke for Saadant ? Nu vistnok , Ondt er Ondt og i og for sig aldrig noget Gode . I og for sig er det Onde noget Unaturligt og derfor aldng Noget , hvorfor man kan takke . Men over denne Verden , fuld af Lidelse og Smerte , hersker den almcegtige Frelser og vil vende endog den bitreste Smerte ttl himmelsk Sodhe6 for Sine . ' Paulus takker hellerikke i Fcengslet derfor , at Folkene i Filippi have mishandlet ham saa arusomt , men for , at Gud ganske vist har ladet dette Onde ste kun for at berede sin fanane Tjener og sin Menighed paa Jorden Frelse deraf . Saaledes takkede de i Troen , og baaefter fik de ogsaa skue det . Saaledes kunde Josef takke , ikke paa Grund af sine Brodres Slethed , men trods den . Af den Tro , at Gud ingen Fejl kan ajore , men maa vende Alting til det Gode for Sine , — af denne Tro kom denne Tak , og i sin Tid blev den ogsaa herlig kronet . I denne Forstand kan du ogsaa takke for en forloren Son . Naar du ellers ogsaa trofast gjor , hvad du kan for ham , saa skal din Gud til sin Tid , var det end forst i Evigheden , <b>vise</b> dig , at din Tak var idel <b>Visdom</b> , og at du ikke tiltroede hans Magt oa Kjcerlighed for Meget . O , naar vi kun virkelig vilde tro , at Guo ved Jesus Kristus er vor Fader , og at vi virkelig ere haus Born , — vi vilde da mer og mer lcere at vcere taknemmelige for Alt .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111400039'target=_>Nissen, Johan, 1863, J. Nissens Samtaler over Den bibelske Historie</a><h4>272</h4>( paa Adam igjen ) med £ jøb . Og tcbtc hende til ham , bebber det kortelig iV . 22. Vi behpve ikke at bære bange for , at tåge Ordene her ganske som be staae . Gud selv har ført <b>Kvinden</b> hen til Manden og har , som de Gamle smukt og rigtigt übtrbjte det , bæret ben fyrste « rubefører , og saaledes Vennen , Joh . 3 , 29 , der vist HMgen har L ^ det stg over tßrudgommens t Brudgommens SRøft . Her kunne nu ogsaa Ordene i C . 1 , 28 Bebft tnbffbbeø : og ledte hende til ham , og Gud saade til bem : ( dette er Indvielfen af bet forste Wgteskab ) Border frugtbare og mangfoldige og opfylder Jorden og gjsrer den Eder underdanig og regjerer over Havets Fiste og over Himlens Fugle og over hvert Dyr , som kryber paa Jorden ; see jeg har givet Eder alle Urter , som give feæb , som ere over al Jorden , og allehaande Trceer , i hvilke erTrceers gritgt , som have @ ceb ; de stulle vcere Eder til Fsde . C 2 23 - Adam sa ^ de : benne Gang er det Been af mine Veen og Kjod af mit « løb . Da Adam vaagner af sin Spvn , seer han den Forandring , ber er foregaaet med hans Legeme , idet en tidligere Deel deraf var adskilt derfra . Pan veed Intet om <b>Kvindens</b> Skabelse , men han veed liae da han seer Eva , at han t hende sinder , hvad han føgte , veed , at hun er hans Lige , ja veed , at hun er tågen af Manden . Han giver ogsaa hende Navn , ligesom han tidligere har givet Dyrene Navn . Her have bt en Prpve paa den Maade , Adam giver Navne paa : Man din de * ) ITIk ^ JI tågen af Mauden . Hvor dybt stuer dette Navn md i sEgteflabets sande Vcesen , i dette Menneskets fyrste Forhold til Mennestet! Hvis man havde spurgt os , de sidste Dages kloge « ørn , os Dannede , om et Navn , hvilket vilde vi da have gibet ? — V 25 - be bare begge nøgne Adam og hans <b>Hustru</b> , og bluedes ikke ' ; hermed betegnes Begges Tilstand som en sand Ustyldigheds Tilstand idet Skammen , saaledes som fenere efter Faldet , er Bevidstheden om en mdtraadt Forstyrrelse og bet legemlige Tegn herpaa . - Saaledes haandhcever altsaa © frtften Mennesteslcegtens Nedstammen fra eet Par Hermed stemme vore bedste Naturforskere overeens . A . v. Humboldt ' Kosmos , Entwurf einer physischen Weltbeschreibung , B . 1 , S 385-li^t . s " ^ ^ " ^ 7 Mennesteslcegtens Eenhed , modsige bi ogfaa hiin hbet glcrdelige Antagelse af Søtere og lavere Menneskeracer . Der aives Folkeftammer som ere mere skikkede til <b>Dannelse</b> , Søtere uddannede og foræblebe ved <b>aandelig</b> Cultur , men ingen cedlere Folkestammer " Dette er nu for Lcereren , hvem det maa overlades , hvormeget han » it meddele stn Skole af bet Ovenstaaende . Men tre Ting af Historien maae nøbbenbtg fremstilles i enhver Skole - fra Elementarstolen til den hMste - og fremhceves af bæreren , Det Øørfte : Guds gabers!^. ' ^ ' . H " " staber de fyrste Mennester i sit Billede , planter Hav n for bem viser fig for dem og taler med bem , tåger dem i fin f ok ( Yver Adams <b>Aand</b> , idet han maa give Dyrene Navne ) , advarer dem for skadelige Ting , stiller Adams Lcengsel efter en Mage knytter Begge til tønanben ved ben cegteskabelige Kjcerligheds snevreste Baand

<h4>379</h4>heden gjorde det ,  Hvis saadan Kjcerlighed de kjendte , Alle sig til Christum vendte " ( No v alis ) . Al Jorden havde eet Tungemaal og eens Tale , V . 1. Med disse Ord viser Skriften hen til et Grundsprog . Naar vi troe paa Menneskestcegtens Nedstammelse fra eet Par , saa ville vi ogsaa finde et Grundsprog meget naturligt . Dette kan ifolge Skriften ikke have vceret noget andet end det hebraiske , fordi det hebraiske Sprog gaaer lige op til Begyndelsen ved Navnene Adam , Ischa ( Mandinde ) , Eva o . s . v. Da nu Menneskeslcegtens Tilstand var reen og simpel , maa ogsaa Sproget have vceret det . Vi have allerede ogsaa havt et Veviis paa , hvor dybt Adams Sprog formaaede at trcenge ind i en Tings sande Vcesen , da vi betragtede , hvorledes Adam giver Tingene og sin <b>Hustru</b> Navn .  Det hebraiske Sprog er af en meget fast og simpel <b>Dannelse</b> ; i alle Gjenstande , som angaae Menneskenes Forhold til Gud , af en uendelig Rigdom ; * ) et meget aandeligt Sprog , der npie slutter sig til Tanken , med faa Ord , som efterligne Lyden ; men übpieligt for det practiste Liv og uskikket til den blot forstandige Tankebevcegelse . " Gerlach , A . T . — Sproget overhovedet er en Guds Gave . Han tog allerede ved Legemets <b>Dannelse</b> Hensyn til Sproget .  Til Sproget ( siger W . von Humboldt , vor Tids Mester i Sprog ) passer Menneskets opreiste Gang , som er Dyrene negtet : thi Talen vil ikke lyde dcempet nede ved Jorden , den forlanger at strymme frit fra Lceberne til Bret , at blive ledsaget saavel af Udtrykket i Blik og Miner som af Bevcegelse med Hcenderne , og tillige at omgive sig med Alt , hvad der betegner Mennesket som Menneske . Mennesket aftvinger sine legemlige Redstaber den artikulerede Lyd , al Tales Grundlag og Vcesen , formedelst sin Sjcels Trang , og Dyret vilde kunde gjyre det Samme , naar det var besjcelet af samme Trang . " Saaledes viser Sprogets Oprindelse sig ogsaa i Skriften . Adam har Ingen at tale med , da han giver Dyrene Navne , men hans <b>Aand</b> driver ham til ogsaa udvortes ved Sproget at fremstille det , som han indvortes stuer . Sproget er en Deel af Aandens Naturhistorie , siger Humboldt , deraf fslger , at der hos Dyrene egentlig slet ikke kan vcere Tale om Sprog . — Naar Sprog et overhovedet er en guddommelig Gave . saa ere S pro gene mere menneskelige . Et Folkesprog er Aabenbaringen af et Folks <b>Aand</b> , ligesom Individets Sprog udvortes fremstiller dets <b>Aand</b> . Alle Sprog ere aldeles organiske ( indenfra udåd ) , og ligesom alt organist Liv og Bevcegelse tiltaler vor <b>Aand</b> , saaledes ogfaa Beskjeftigelsen med Sprogene . De uendelige Mangfoldigheder * * ) deri viser den menneskelige Aands Herlighed og Dybde . Men i denne Sprogenes Mangfoldighed fremviser Skriften os en Skyggeside , som Videnskaben ikke veed Noget om . Naar den efterviser Oprindelsen til denne Mangfoldighet , i Taarnbygningen i

<h4>517</h4>meget vcerd ; det <b>vise</b> ogsaa de Gaver , som han gav Rebekka , og V . 53 : Han udtog Guldtoi og Klceder . Abraham havde betroet ham Meget , men Tjeneren udleverede , hvad han skulde og beholdt ikke Noget for sig selv . LErlighed . Dit Herstab maa ogsaa give Dig Meget mellem Hcender . Ncevner Noget , <b>Piger</b> ! hvad faae I ? Linned , Sylv- Wi , Penge , Huusgeraad . Hvad faae Lceredrengene ? Tjenestekarlene ? Siger og tcenker da som Abraham engang : Jeg vil ikke tåge en Traad heraf ; — jeg vil ikke beholde saameget som en Stilling . Exempler : Naar <b>Piger</b> kjpbe noget paa Torvet ; Karle fortcere noget underveis ; Lceredrenge og Butitsvende scelge Noget . Saadan LErlighed kalder man ogsaa Troflab . Med Ordet Tros ka b betegner man dog ogsaa Alt , hvad en Tjenende stal gjsre mod sit Herstab . 1 Cor . 4 , 2 : Is vri gt udkrceves af Huusholdere , at de maae findes troe . Vcer tro i din Tjeneste , deri ligger det nylig omtalte Tredobbelte : Lydighed , Kjcerlighed og LErlighed . Gjentagelse . Den Tjenende i Forholdet til Gud . Hvorledes der ? til Mennesker ? 1. 2. 3. I stulle vcere chriftelige Tjenestefolk . Et Par vigtige Bibelsteder anfsres her , som have Hensyn hertil . Eph . 6 , s — B : Eders Herrer ester Kjsdet , — som kun have Magt over Legemet . Lydighed med Frygt og Bceven , — med Alvor og Flid ; i Hjertets Eenfoldighed , — ikke paa en listig Maade , scette et X for et U , som man siger , uden Falsthed . Nientjeneste , — Tjeneste for Ens Mne , kun Trostab , naar Herstabet seer derpaa ; som Christi Tjenere , som om I tjente den Herre Jesus . Ja , saaledes som Du tjener dit Herstab , saaledes tjener Du ogsaa Jesus . Det frembringer Villighet » , V . 6. Hvorledes det ? Christi Kjcerlighed tvinger dertil og faaer engang Naadelpn , V . 8. Tit . 2 , 9. 10 : Vcere underdanige ; velbehagelige — altsaa Kjcerlighed; ikke sige imod ; ikke besvige — vcere cerlige ; pryde Guds <b>Lcerdom</b> — at man kan see , hvorledes Evangeliet frembringer lydige og troe Tjenestefolk . 1 Pet . 2 , 18. 19. 20 : Adlyde ilke blot de gode Herrer , men ogsaa de fortrcedelige , de urimelige . Hvorfor ? V . 19 : Thi det sinder Naade ( hos Gud ) , naar man fordrager Gjenvordigheder og lider Uret . 0 Bsrn , mcerker Eder det Sidste . Mennester sige ofte til en <b>Pige</b> eller en Karl : Det taaler Du , det lader Du Dig byde ? Gud taler her paa en anden Maade . Hvorledes lyd nu Ordene ? Lcerer dem , bevarer dem i Eders Hukommelse og tcenker paa dem , naar I trcrde 1 Tjeneste . En lille Historie til videre Benyttelse . Lene var meget dygtig ; men hun bildte sig ikke saa lidt md af sin Dygtighed . Hendes <b>Moder</b> fcestede hende til en Bondekone og sagde ved Affleden : Bed Gud dag < lig om , at han vil give Dig Lykke og Velsignelse i din Tjeneste . Men Lene sagde : Jeg er slet ikke bange , jeg stoler paa min Dygtighed . Strax den fyrste Morgen stulde Lene gjore Ild paa og cmstrcengte sig en halv Time forgjceves for at staae Ild ; endelig lpb hun ind til Nabokonen for at hente Lys . Men hun gled paa Isen og flog Lygtcn itu . Da fik hun allerede den forste Irettesettelse . Hun undstyldte sig imid « lertid med , at Fyrsvamven var bleven fugtig af Tsveiret , og at der

<h4>1758</h4>velbehagelig og velsignet fremfor alle <b>Kvinder</b> ? Det er Idmyghed . der tenker ringe om sig felv , ikke gjor sig til af Noget og forskrekkes , naar den bliver roest . Et saadant ydmygt Hjerte havde Maria . Og — de Idmyge giver Gud Naade . Frygt ikke Maria , thi Du har fundet Naade hos Gud . D . e . din Roes og din Heder ere ikke dit Vcerk eller din Fortjeneste , men en Naade , d . e . en ufortjent Gave . Da Maria ogsaa behøver Guds Naade , faa er hun altsaa ogsaa syndig , og derfor , hvor.hsit vi end cere den , som en Engel cerer , saa tilbede vi hende dog ikke eller bede til hende , saaledes som Katholikerne gjore . Hvor.ledes skal dette gaae til ? Da Engelen havde sagt hende den store Hemmelighed : See , Du skal undfange — , da sporger hun : Hvorledes skal dette gaae til ? Sagen selv troer hun , men hvorledes dette saa Usedvanlige stal stee , kan hun ikke forklare sig . Her staaer Maria hsiere end Zacharias ; hun er troende , som Elisabeth ogsaa priser hende for : Salig er hun , som troede . Denne Tro opfylder hende ogsaa med Glede , saa hun lovsynger Gud . See , jeg er Herrens <b>Tjenerinde</b> , mig skee efter dit Ord . Dermed vil hun sige : Jeg hengiver mig ydmygt og lydigt ( <b>Tjenerinde</b> ) i Guds Villie , han gjore med mig , hvad han i sin Viisdom og Naade vil . Det er Hengivenhet , og Taalmodighed . En saadan Rolighed og Tilfredshet » med Alt , hvad Gud gjer , pryder iser et Kvindesind . Hvor Gud mig forer , er i Tro og Haab mit Hjerte stille , i mig hans egen Kraft vil boe , hvad kan mig fra ham stille ? Thi fatter jeg et trsstigt Mod , fordi Guds Vei er altid god , ja vist den allerbedste . Ps . " Nr . 48. * ) Marias Lovsang , V . 46 — 55. Marias Herlighed og Hpihed viser sig endelig i den deilige Lovsang hos hendes Veninde Elisabeth . Denne hendes Herlighed er : hendes <b>Kundskab</b> til Guds Ord og trofaste Bevarelse deraf og hendes taknemlige Hjerte , der udtaler sig i en Lovpriisning af Gud . Hendes Lovsang har stor Lighed med Hannas , Samuels Moders , og hun har vistnok kjendt den . Dernest er hendes Hymne ogsaa fuld af Ord as det gl. Test . , som I kunne see af de mange Bibelsteder under hvert Vers . Min <b>Sjel</b> ophpier Herren , d . e . hun vil gjore Gud ophoiet og stor i sin <b>Sjel</b> , nemlig ved sin Lovsang . Min <b>Aand</b> fryder sig i Gud , min Frelfer . Ja , Ingen fortjener mere end Gud , vor Frelser , at vi fryde os over ham , thi han er den Bedste og Kjerligste , den Megtigste og Herligste . Naar Gud meddeler sig til Nogen , som her til Maria , maa det fromme Sind opfyldes af en inderlig Hjertensglede ; thi vi ere Syndere , og han er Frelseren . — Han har seet til sin Tjenerindes Ringhed . Hun tenker paa sin Ringhed . Thi ringe var hendes Stand , hvorledes ? hendes ydre Stilling ; hendes Hjerte , hvorfor ? Mangen Anden havde

<h4>2616</h4>Apostlene have  seet hans Herlighet » " , og hvad de have hort med deres Dren og seet med deres Dine , og hvad deres Hcender have fylt paa , det forkynde de os , at ogsaa vi stulle have Samfund med dem i Christo , 1 Joh . I , I — 3.1 — 3. Eenfoldigt og jevnt , uden Reflerion have de beskrevet Jesu hellige Liv ; forat kunne efterfolge dette ( ikke for et Tanteexperiments eller en Digtnings Skyld ) have de offret Gods og Blod og Liv ; og i Aarhundreder har aldrig Nogen fundet en Plet paa dette Liv . Lcider os nu engang betragte Frelserens Liv ncermere . Herren var hellig og reen for Synd . Han , der kjender sig selv bedst , og i hvis Mund der ikke var Svig , siger selv : Hvo af Eder kan overbevise mig om nogen Synd ? Apostlene vidne , 2 Cor . 5 , 21 : Den , som ikke vidste af Synd , har han gjort til Synd for os . 1 Pet . 2 , 22 — 23 : Hvilken ikke har gjort Synd , der blev og ikke funden Svig i hans Mund — . Hebr . 7 , 26 : Thi saadan en Ippersteprcest sommede os , som er hellig , uden Skyld , übesmittet , adstilt fra Syndere og ophoiet over Himlene . Derved bliver Christus , hvad intet andet Menneske kunde vcere . et fuldkomment Forvillede for os ,  at vi stulle efterfolge hans Fodspor " . Christus er et Monster for alle Stcender , skjondt han f . Ex . ikke har levet i sEgteskab , fordi det her ikke kommer an paa enhver enkelt Pligt , men paa den <b>Aand</b> , hvoraf Pligten fremgaaer . Jesu Liv kan lcere os Alt , sagde allerede de Gamle , og Ullmann siger med Rette i sin Bog : Die Sundlosigkeit Jesu , hvoraf vi her have benyttet Meget : Christus bliver ikke stillet os for Bie , forat vi skulle copiere ham , men forat han stal leve i os . — Christus er et Monster for Manden saavelsom for <b>Kvinden</b> ; thi han forener i sin Charakteer den Edleste Mandigheds og den reneste <b>Kvindeligheds</b> Dyder . Hau er som heltemodig Mand kraftig uden at bruge uogeu udvortes Magt , og paa den anden Side er han mild og blid som den blideste <b>Kvinde</b> , hvor han kan og stal hjcelpe , troste og <b>vise</b> Medfolelse . Vi ville her anfore nogle smukke Steder af Ullmann , der fremstille lefu Charakteer i Almindelighed .  lesu Liv gjor Indtryk af den hoieste Ro , og Besindighet » , forbunden med en levende og dyb Begeistring . Hans hele Vcesen er Ro og Fred , og de gamle Propheters opblussende og sortcerende Ild forvandler sig hos ham til en uafbrudt gudopfyldt Sjcelsstemuing . Han vandrer i den aandelige LEther som i sit eiendommelige Livselement , hvortil Andre kun ha ; ve sig i enkelte salige Nieblikke . Ligesom Solen paa det rene Firmament gaaer han stille og fast og uden at afvige fra sin sikkre Bane , udbredeude Lys og Liv . Hans Handlinger ere fulde af Kjcerlighed , uden opbrnfende Affekt , uden Heftighed og Lidenstab . Han gjyr Intet übesindigt og uden Heusigt ; Alt , hvad han begynder paa , steer med Sikkerhed og naaer sit <b>Maal</b> ! Selv hvor han trceder straffende op i Ord eller Gjerning med alvorlig Uvillie , er det aldrig en opirret Stemning , der giver sig Luft , men stedse Kjcrrlighedens hellige Vrede , der hader Lasten , men elsker Mennesket i den Lastefulde ; og aldrig overskrider han Maadeholdets rette Grcendser . — Jesus er mild og blid ; overalt opsoger han de Ringe og Foragtede og dem , der trcenge til Hjcelp , og nedlader sig frivilligt til den dybeste Fornedrelse og vcerste Lidelse ; men ud af Fattigdommen og Elendigheden , som han har ifort sig , fremstinner i ethvert Forhold i hans Liv en hoi og kongelig <b>Aand</b> .

<h4>2884</h4>1 ) Paulus for sin Omvendelse . Saulus havde allerede i Jerusalem forfulgt de Christne . Han siger om sig selv , C . 22 , 4 : Jeg bandt og overantvordede i Fcengsler baade Mcend og <b>Kvinder</b> ; V . 19 : Jeg hudflettede i Synagogerne dem , som troede paa Dig ; C . 26 , 10 : Naar de bleve tagne as Dage , gav jeg min Stemme dertil . Hans Raseri mod Christendommen sammenlignes i V . 1 med et vildt Dyrs Fnysen . Han meente , C . 26 , 9 , at han burde gjore Meget imod Jesu Navn . I 1 Tim . 1 , 13 siger han om sig selv : Jeg var tilforn en Bespotter og en Forfplger og en Forhaaner . Derfor var Forfplgelsen i Jerusalem ham ikte nok . Han lod sig give Breve til Synagogerne i Damascus i Syrien , V . 2 , paa det , om han fandt nogle Mcend eller <b>Kvinder</b> , som vare af denne Lcerevei ( d . e . som fulgte denne Vei til Salighed , denne Religion ) , han da kunde fere dem bundne til Jerusalem ; og han var allerede ncer ved sit <b>Maal</b> , uden at hans Sind havde forandret sig . Hvorledes kunde han dog gjore alt dette ? 1 Tim . 1 , 13 : Jeg gjorde det uvittcrligt i Vantro . Uvidenheden er altsaa tilregnelig , som Tilsetningen : i Vantro , ogsaa viser . Men selv om en forskyldt Uvidenhed er Aarsag til Synden , saa er dog Redning endnu mulig ; og naar den ikke kom fra Panlus , fra hvem kunde den da alene komme ? Fra Herren ; og fra ham kom den ogsaa ; 1 Tim . 1 , 16 : Jesus Christus vilde paa mig , som den storste ( Synder ) , <b>vise</b> al Langmodighed, dem til et Exempel , som skulde troe paa ham til det evige Liv . Denne Begivenhed angaaer altsaa os ; og hvorfor ? Her ville vi ogsaa anfpre Ordene i Rom . 2 , 4 : Foragtcr Du hans Godheds og Taalmodigheds og Langmodigheds Rigdom og veed ikke , at Guds Godhed leder Dig til Omvendelse ? Hvorledes see vi denne Taalmodigheds  Rigdom " i Panli Liv ? og hvorledes erfare vi selv denne Rigdom ? Naar vi saa ofte lyve og hcevne os og spre ugudelig og skammelig Tale , og han saa dog lader os ustraffede . Hvorfor steer da dette ? Fordi hans Godhed vil lede os til Omvendelse . — Da nu Paulus gaaer videre og videre paa den betraadte Bane , saa forbarmer Gud sig over ham . Han hyrte en Rpst , der sagde ( i det hebraiske Sprog , C . 26 , 14 ) : Savl , Savl - ; og tillige saae han Herren , der strax gav sig tilkjende for ham som Jesus . Dette fortcellcr Varnabas Apostleue i V . 27. Ogsaa senere har Paulus feet Herren , C . 22 , 17 : Det stete mig , der jeg kom tilbage til Jerusalem og bad i Templet , at jeg faldt i Henrykkelse og saae ham . C . 23 , 11 : Men Natten derefter stod Herren for ham og sagde o . s . v. Derfor kuude Paulus ogsaa sige , 1 Cor . 9 , 1 : Er jeg ikke en Apostel ? Har jeg ikke feet vor Herre lesum Christum ? 1 Cor . 15 , 9 : Sidst af Alle blev han feet af mig , som det utidige Foster . Han mener her Aabenbarelsen ved Damascus . Endnu en Aabenbarclse af Herren ( man veed ikke , hvornaar den er

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009072703031'target=_>Hofacker, Ludwig, 1863, Prædikener paa alle Søn- og Helligdage</a><h4>1517</h4>bekymrede ikke den kananceiste <b>Kvinde</b> . Da hun erfarede , at Jesus var i Ncerheden , sta opsogte hun ham , übekymret om , hvad hendes Naboer , hendes Slcegtninge vilde sige . Med stor Hastighed lob hun ester ham udenfor Tyri og Sidons Egne . Hun har neppe faaet Bie paa ham i det Fjerne , sta oplsfter hun sin Stemme og raaber sta godt hun kan :  Herre , Davids Son , forbarm Dig over mig , min Datter plages ilde af Djcevelen ! " Ak , hvilken Skam vilde det ikke vcere iblandt os , dersom der hos en herværende Synder opstod en stadan Iver ester at soge lesum , dersom man saa tydeligt kunde mcerke det paa ham : dette Menneske soger lesum . Og dersom der nu engang paa fri , aaben Mark , hvor Enhver kunde hsre og ste ham , paakom ham Trang til at falde ned paa sine Knce og raabe : Herre , forbarm Dig over mig ; jeg plages ilde af Djcevelen : ikke sandt ? dette vilde vcere en Skam for den hele Familie ! Hvor mcerkvcerdig vilde man ikke finde en saadan Tildragelse ? Hvilke spottende Bemcerkninger vilde man ikke gjsre derover ? De Fleste vilde udgive et stadant Menneske for at vcere forrykt , fordi han håndlede anderledes end alle kloge Folk . Andre igjen vilde mene , at dette Menneske maatte have noget Scerdeles paa sin Samvittighed , da han var i en saadan Hjertens Nod . Nei , I blinde Pharisceere ! man behover ikke at have noget Mord paa sin Samvittighed , for at <b>vise</b> sig staledes. Hvo har da viist Eder at flye fra den tilkommende Vrede ? En saadan Bod , en saadan Nedsynken paa Eders Knce , et stadant Raab til den Herre lesum er hoist nodvendigt for Eder alle , og dersom I ikke lcere at skrige staledes , saa stal den Almcegtiges Vrede fortcere Eder paa Vredens Dag . Men Hedningerne skulle optrcede paa hun Dag og fordsmme den lunkne Chriftenhed; Kafferne ville gjore det . Da Missionceren Read i Aaret 1816 prcedikede Evangeliet for dem , sta gik hans hedenske Tilhorere, bver Gang han havde stuttet sit Foredrag , tilside , den Ene bag en Bust , den Anden bag et Trce , kncelede ned , fremforte hvad de havde hort , i en Bon og ssgte Lys for deres formsrkede Hjeiter fra Herrens Ansigt . Hvor har man hort noget Lignende hos os ? Vi burde stamme os , vi stolte , sikkre , dorste Christne ! Det er allerede et Tegn paa , at et Menneskes Tro er af den rette Art og har en guddommelig Livsdrift i fig , naar et stadant Menneske aflcegger den utidige Undstelse og Mennestefrygt og enfoldigen lader sig drive af sin Tro og felger Guds Ord . 2 ) Den kananceiste <b>Kvindes</b> Tro var ogsta stor , fordi den kjcempede sig igjennem alle Forhindringer og enhver Modstand og naaede sit <b>Maal</b> . Hun maa fsrst opssge Frelseren , der vil holde sig skjult . Hun sinder det Huus , i hvilket han opholdt sig , hun gaaer derind og horer , at han allerede

<h4>1529</h4>blivende Forandring i Hjertet ; man paaberaaber sig maaskee sin Bmneret , naar Sandhedens <b>Aand</b> er nerved at negte Hjertet de smaa Hundes Ret , og man bliver saaledes mere og mere forherdet mod Guds Aands Stemme ; man bliver fra Dag til Dag mere uskikket til at gjsre Bod . Hvor sijsn er derimod ikke den kananeiske <b>Kvindes</b> Tilstand ! Hvor aabent ligger ikke hendes Hjerte for Frelseren ! Hvor ydmyger hun sig ikke under alle Guds Rettigheder , og bliver dog ikke forsagt ! Hun kaster sig i Stsvet som en Orm , hvilken Frelseren kan sondertrede ; men han ssndertteder hende Me . Thi den Herre , som boer i det Hoie og Hellige , han er hos de Ssndecknufte og Fornedrede i Aanden , til at gjsre de Fornedredes <b>Aand</b> levende , og at gjore de Sonderknustes Hjerter levende ( Gs . 57 , 15 ) , og den , som ikke har Gjerninger, men troer paa ham , som retferdiggjsr den Ugudelige , ham tilregnes hans Tro til Netferdighed ( Rom . 4 , 5. ) — Lader os nu 2. kortelig see , hvorledes denne Tro er bleven til hos den kananceiske <b>Kvinde</b> . I kunne tenke Eder , at man ikke paa engang kommer til et saadant Sind ; det voxer heller ikke frem af Kjsdet , men er en Guds Gave , som han tilbyder Synderne paa Lydighedens og Lidelsernes Vei . Af vort Evangelium lader der sig imidlertid fremheve noqle Punkter , som give os et temmelig godt Lys angaaende den Maade , paa hvilken den kananeiske <b>Kvinde</b> er kommen til denne Tro . Det fsrste Punkt , som vi i denne Henseende ville merke os , er : Den kananeiske <b>Kvinde</b> havde hsrt om lesum og opfattet Ordet om ham dybt i sit Hjerte . Dette kunne vi allerede see deraf , at hun bad Frelseren om Hjelp for sin Datter , da denne plagedes ilde af Djcevelen . Dersom ikke Efterretninger om Frelserens Undergjerninger vare trcengte frem til hende , > saa havde hun jo aldrig nogensinde gjort dette . Men vort Evangelium giver os endnu videre Fingerpeg . Hun kan ikke alene paa en oveifladisk Maade have hsrt om ham , hun maa ogsaa have forhsrt sig nennere angaaende ham . Dette kunne vi mceike deraf , at hun raabte ester Frelseren :  Herre , Davids Son ! " Hun var en hedensk <b>Kvinde</b> , og vidste saaledes af Naturen Intet af den Forjettelse , som var givet David , og alligevel kalder hun Frelseren :  Davids Son . " Hvorledes er hun vel kommen til denne <b>Kundskab</b> ? Ene og alene ved de noiagtige Efteiretnwger , som hun indhentede ; thi saadanne noiagtigere Betegnelser blive sjelden meddeelte ide fsrste Fortellinger . Da Rygtet om den nye jsdiske Propdet trengte frem til de nerarendsende hedenske Lande , saa hed det vel : der er opftaaet en Undergjorer ; det er en Mand fra Nazareth , han har allerede helbredet mange Syge ved sit Ord , uddrevet mange Djevle ; i Kana har han forvandlet

<h4>1532</h4>Vand til Viin o . f . v. Videre end til saadanne ydre Kjendsgjerninger naae ikke lettelig de Fortellinger , som gaae i Folkenmnden. Men dermed kunde den kananeiste <b>Kvinde</b> ikke finde.sig tilfredsstille ; hun forhorte sig videre ; hun hsrte , at han nedstammede fra Davids Slc ^ gt , og maaskee var den forjettede Davids Son ; hun sogte Underretning om hans Opfsrsel , om hans Hjertelag, og horte om hans Kjcerlighed til de Elendige , om hans store Barmhjertighet » . Og hvad hun saaledes hmte om ham , det frembragte hos hende en übegrendset HsiagtM for Jesus og en urokkelig Tiltro til hans Hjerte . - O , mine kjere Tilhsrere ! Hvad stulle vi sige om denne hedenske <b>Kvinde</b> , som saaledcs annannnede og benyttede de ringe eg upaalidelige Etterretninger , hun havde om Jesus , at den stauke Tro , som Frelseren selv beundrede hos hende , kunde voxe frem af dette lille Frokorn ? Hvad stulle vi sige om denne Kjcerlighed til Sandheden , om denne Trostab i at bevare , hvad hun havde modtaget? Sandel ig ! Denne <b>Kvinde</b> er sat som et lysende og fordsmmende Tegn for os og denne Tidsalders Mennesker . Mange Hedninger , som dog ikke kjende Guds Vei , <b>vise</b> for ncervcerende den storfte Begjerlighed ester Guds Ord og ester Lcerere . Folk , som fsrst for kort Tid siden have lert Evangeliet at kjende , lere med den stsrfte Begjcrrlighed at lese , for at kunne lese Guds dyrebare Ord ; de agte Guds Ord for sin dyrebareste Skat ; de kunne ikke blive mette af at hore om sin Jesus ; de soge Frelseren ; de soge hans Ansigt ; med mange Bonner og Taarer ssge de Guds Naade og sine Synders Forladelse i Lammets Blod . Herrens Herlighed opgaaer over mange Hedninger , og vi ste dem vandre i dens Lys . Vi derimod have Guds Ord i saa rigeligt <b>Maal</b> ; vi have det hjemme og i Kirken , og agte denne dyrebare Skat som noget Almindeligt , og det forekommer os , at det saa maa va > re . Sandheden , det evige Evangelium have vi i saa rigeligt <b>Maal</b> , og vi rsre os ikke , og de Fleste blive i sine Synder, i sit Liv ester Fedrenes Viis , i sin aandelige Uforstand og Blindhed , og takke ikke for Guds ufortjente Forbarmelse , oH tenke ikke paa at takke . Hvad vil dog Gud maatte- foretage sig med os ! Naar han lader sit dyrebare Evangelium predikes for os ; naar han lader Christum afmale for vore Bine ; naar han lader et saa rigeligt besat Bord fremsette for os , og vi ikke aabne Munden for at nyde , men for en Deel endog vemmes ved denne Spise , medens mange Tusinde vilde vere glade ved de Brsdsmuler, som falde fra vort Bord ; hoad vil da Gud blive rwdt til at foretage med sin dovne , kolde , mette , stolte Christenhed ? Det vil han maatte foretage med den , at han aldeles omstoder dens Lysestage , hvortil d ? r i vor Tid allerede er Udsigter , saa at

<h4>2955</h4>Dette er da det jordiske Sind , disse ere de elendige Felger , dette er deres Skjcebne , som hcrnge sit Hjerte ved det Timelige og Forgjcengeliqe , enten det nu er timelige Goder , eller Glceder , eller Hceder . Men ikke meget bedre ere 2 ) de , som ville dele sig mcllcm Verden og Gud , de , som have et de elt Hjerte . Man har Indtrvk as Gud og hans Sandhed , og vil dog ikke forlade Verden : man har vel nu og da erkjendt Evangeliets Krast ; folt , at det er godt at overgive sig iil Herren og handle efter hans Villie ; men Gjenfodelsens Sced , Guds Ord , er faldet blandt Torne , oa. ved Siden as Lamgslen efter at blive salig hcever sig mangeslags Tragten efter Rigdom , 3 Ere , Magelighed , ct sorgfrit , behagelig ! Liv , som man med al Magt onsker at forbinde med sin Tragtcn efter Himmelen . Nu strcrber man da paa alle mulige Maader at give sine urene Omsorger , - sine usle Begjæringer et cerbart , christeligt , pligtmcrssigt Anstrsg , stjuler sin hemmelige Gjerrighed under det Kald , man har at ssrge for <b>Kone</b> cg Bsrn eller for det almindelige Vedste ; sin forfengelige AdspredelscZsyge under den foregivne Pligt , at leve overeensstemmende med sine Forholde , at omgaaes sine Medmennesker paa en venlig og forekommende Maade overeensstemmende med den almindelige Menncskckjcrrlighed , og man tilfoier , at man ved at drage sig tilbage fra saadanne tilladte Fornoielser kan skade sit Rygte , ja vel endog give Forargelse ; sin LErgjerrighed tildcekker man med et Figenblad , som man kalder <b>Uddannelse</b> af sine Evner , Gavnelyst og Iver for det almene Bedste ! — Man har vel Noget , som ligner Gudsfrygt , men det er ikke den rette , men en Trceldomsfrvgt, som kommer af , at man har en ond Samvittighed for sin Bienskalkhed ; man horer vel gjerne noget Godt , men det faester sig ikke ; det folges ikke af noget levende Bifald , nogen Tilvant i Lys , Liv og Erfaring ; man forglemmer det hurtig igjen , og naar man en Stund har vauet from med de Fromme , lever man igjen verdsligt med denne Verdens Bsrn ; man har vel nogen Lcrngsel efter <b>Sjelens</b> Frihed og Salighed , men man vover Intet derfor ; ftygtsom regner man efter , hvormeget man maa give Slip paa , tabe , fornttgte og taale , hvis man beslutter for fuldt Alvor at overgive sig til Herren ; man skjcelder sig selv ud for en Synder , og tåger ogfta el Tillsb til Omvendelse , men man hader ikke Synden , og klager ofte over indre Fiender , med hvilke man dog staaer i det beiste Venskab , dersom man vil ste noie til og aflcegge en redelig Bekjendelse , ja hvilke man bevarer som sin Biesteen . Man vilde gjerne tilhore Chnsto , men man kommer dog ikke til ham , fordi man bestandig har saa mange andre Sager at vetamke og ftrge

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112101047'target=_>, 1857, De fire Evangelier og Apostlernes Gjerninger</a><h4>361</h4>gik i Opfyldelse , vil med disse Ord vlstnok blot antyde , at Es . 7. 14. fuldstcendig blev fuldvyrdet , eller at le . sus virkelig blev fodt af cn , , lomfr , , " ; derimod vil han ikke tilkjendegive Noget angaaende losephs og Marias crgteskabcligc Samliv eftcr Jesu Fsdsci, eller om de senere bave havt Born med hverandre . Ordet  indtil " beviser, paa Grund af Matthcei vvenncevnte bestemte <b>Formaal</b> , i denne Krnseende aldeles Intet , og ssrstefodte" kaldtes . ifslge Loven < 2 Mos . 13.2 . ) , Alle ,  som wrst aabnede Modcrlivet," ogsaa om Moderen kun b : vde dette ene Barn ( 2 Mos . 22,29 . ) . Dette Vers taler saalcdes hverken for eller imod den Antagelse , at de , som i det N . T . kaldcs  lesu Brsd re " , virkelig ere hans Halvdrodre , Bsrn af Jesu <b>Moder</b> . Dog maa det indrsmmes , at et blot Skin « ZEqtestab ingenlunde var i Overensstemmelse med den Tids <b>Aand</b> , og at Evangelisten , om det ogsaa for ham bavve lagt noget Utcenkeligt i loscphs agteskabelige Forening med Maria , jo ' saare let kunde have sagt :  ban boldt sig aldrig til bende . " Sagen bliver faaledes dunkel , og man bsr ikke lcegge nogcn overdreven Vcegt hverken Paa den ene eller den anden Mening . Se forresten herom Matth . 12. 46. A . Om en evig lomfludom og deraf ssydende scrregen Helligbed , som Traditionen allerede fra det 2 det Narbundrede og senere den katbolste Kirke har tillagt ' Maria , veed den hellige Skrift aldeles Intet .

<h4>4215</h4>med Hensyn til sin Natur en jordisk Fode , bvorvel paa underbar Maade frembragt ved Guds Kraft ; men vistnok var den , som Sindbillede paa GudS ncrrende , forssrgende Trossab . en aan « delig ssode , ja , i en <b>vis</b> Betydning et Sakrament ( 1 Cor . 10.3 . A . ) . Imid < lertid , fordi den dog nærmest kun erncrrede legemlig , og kun paa Grund af denne Erncrrings Peskaffmhed tillige <b>aandelig</b> , fordi den var af jordisk Natur, fremdeles fordi dens egentlige <b>Formaal</b> gik tabt hos de Fleste , for hvem den kun var en legemlig Fode : kunde Christus fremstille den i Modscrtning til det sande , <b>aandelig</b> og legemlig nicettcnde og fornyende Himmelbrod . som den himmelske Fader giver . Httnr Vrsd gav Moses , et Menneske , der var af Jorden og talede af Jorden ( C . 3.31 . A . ) , der ' ikke kunde give Livet af sig selv , eftersom ban ikke bavde det i sig selv ( C . 5.26 . A . ) . Dette Bred deri . mod giver ikke et blot Menneske , men alene Faderen ved den ham lige , evige Son , og dennt Son er selv Brsdet . 3. V . 33.  er det , som " etc. , eg.  den . som nedstiger " etc. Ikke det torste jordiske Menneske , eller nogen af bans Born . ei heller Noget , som ban gav ; men det andet Menneske , Herren af Himmelen , l Cor . 15,4 . . 4. V . 33. De opfattede Jesu Orv kjsdeligt , som den Samaritanste <b>Kvinde</b> , og der ? s Bon blev derfor i samme <b>Aand</b> og Mening , som hendes , C . 4,15 . Uagtet Jesu Ord V . 33. troede de. at det var noget udenfor Jesus , som han vilde give dem .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015010648109'target=_>, 1852, Almennyttigt dansk Konversations-Lexikon over alt det Videværdigste i Naturen, Kunsten og Videnskaben, fra den ældste til den nyeste Tid, i alfabetisk Orden, efter Brochhaus's Tydske Original, nyeste Udgave</a><h4>375</h4>Aar eeldre . Han var ikke uden Godmodighed og meget retsindig , men manglede en <b>vis</b> Bsielighed i Fantasien , som saa godt underftstter den chriftelige OverbcerelseSaand ; nu krcever vi hos de Geistlige Anerkiendelse af de ophsiede <b>Formaal</b> for Poesi og Kunst og andre Arter af den menneskelige Strceben som et Tegn paa <b>Dannelse</b> , ei paa Billighed , i hine Tiber var det anderledes . Det er imidlertid tydeligt , at Schsnheyder , som selv har givet sig af med Salmedigtning , virkelig har glcedet sig ved E . ' S Digte og agtet ham ysit som Digter ; hvilket vift har vcrret den vigtigste Grund til HanS Deltagelse i E . ' S Skiebne . Tonen i HanS Breve er ofte saa moraliserende , at det ikke er at undres over , at E . har lceft en Del af disse med svulmende Selvfslelse og Bitterhed . Hvad der iscer maatte oprsre ham , var et Forstag , Schsnheyder giorde ham om at blive Lem paa et Hospital ! Hertil svarer E . ( 9 de Octb . 1776 ) : Saa velmeent , som jeg troer , at DereS Forstag er , min kicereste Ven , saa har det dog sat og ikke kunnet andet end scette mig i den yderste Bestyrtelse og Angst og det kicereste det eneste kicere jeg finder deri , er nsdvendig det , som De forsitkrer mig , at de har havt den Kicerlighed ikke at tale til Andre om saadanne Ting , fsrend de hsrte min Mening . Men kunde De ikke vide denne Mening ? For den barmhiertige Guds Skyld , lad dog ikte min Tillid til Dem og Deres Omhu for mig gisre mig rent ulyksalig ! Jeg seer det nu , og jeg har seet det fsr , at jeg i Deres Vine kun er et hielpelsst elendigt Krce , som maa takke Gud , om det faaer det daglige Brsd , fsr det kreperer . Jeg er nu vist ilke et saadant Krce . Men det frygter jeg for at vcere , et Almisselem . Gud ! retfcerdige Gud ! har jeg fortient en saadan Idmygelse l " Det var paa Sebcekshus , hvorhen han var blevet flyttet , fordi han havde forelsiet sig i en ung <b>Pige</b> , maasie Fiskemesterens Datter, da han modtog dette Forstag ; men imedens han fslte sig saa grcrndselss ulykkelig , sit han ved Frederik Moltkes Bistand en Gave af 100 Daler fra Arveprinds Frederik og blev noget efter ved hin VenS samt Carstens ' s Bestrcebelser forflyttet til Kisbenhavn , hvor en omhyggelig Pleie og en Smule Paafiionnelse ( han sil nu en lille Pension af Staten ) mildnede hans Lidelser i de saa Aar , han endnu havde tilbage af sit Liv og styrkede hans <b>Aand</b> til ny cerefulde Foretagender . Her dsde han den 17 de MartS 1781. Hans Liv , som i saa mange Henseender var beklageligt , havde dog vcrret egnet til at udvikle HanS poetiske Evner og udruste ham med de for Digteren nsdvendige <b>Kundskaber</b> og Erfaringer . Fra sin egen Erindring kunde han hente sande og levende Trcek til Slildringen af herlige Naturscener , forsiiellige

<h4>1918</h4>Arbeider i Oc-Sproget et Brudstykke af Boethius's Levnet , bearbejdet for at tiene til Opbyggelse, fra Slutningen af det 10 de Aarhundrede, Helgenlegender , som Legender om den hellige AmantiuS , den hellige FideS fra Agen , fra det lite Aarhundrede , Npistnll » l ' aieit « , det er : halv Latinfie , halv Romanfie Kirkesange , ValdenserneS « åndelige Digte i den Piemontefifie Dialekt , fta det 12 te Aarhundrede , de sidste i enrimede Strofer eller ncrsten prosaiske Tirader , ( i RaynouardS : „ Olinix ile » « le « ' l ' ioubaHoulS " og hans Kum»n") og endelig endog kunstmcrsfige Hymner efter Latinfie Forbilleder i kortere VerS , i hvilte Begyndelsen til Troubadourernes Kunstpoesi viser sig . Ligeledes var de fsrsie littercrre Forssg i det Nordfranske Sprog Oversoettelser eller Efterligninger af Latinfle Originaler for det meste af kirkelig-religivst Indhold, som det celdfte rythmifle Mindesmoerke i den Nordfranske Romanzo nemlig Prosaen (Kirkesangen) om den hellige Eulalia ( i „ Lll > l » l , < m . em " , udg . af Hoffmann og Willems , Gent 1837 ) , fra det 9 de Aarhundrede , Overscrttelserne af Kongernes og MatkabcrerneS Boger i Prosa , hist og her med rythmifle Steder , fra det 12 te Aarhundrede , Oversættelsen af den hellige Bernhard ' s Prædikener , fra samme Aarhundrede yugtres livre » cle » roi « , trs > - llult « en 5,9 N9 . " eto . udg . af Lerour de Lincy , Paris 1841 , 4. ) , Oversættelser af nogle af Gregor den Stores Skrifter , Kpi » t < » lN tareitN og Helgenlegender , der er ganske folkelige . Anden Hovedperiode . Foruden den i det 12 te Aarhundrede vakte og styrkede Nationalfslelse havde et politisk - socialt Element saa kraftig udviklet og saa almindelig udbredt sig , at det var blevet en aandig Magt og allerede imod Enden af det 12 te Aarhundrede , altså« ncrsten samtidig med Trangen til en Nationallitteratur og med en saadan <b>Uddannelse</b> af Sproget , at denne kunde fremstaae , begyndte at giore Kirken det udelukkende Herredomme over Tidsaanden stridig og deSaarsag snart blev det egentlige LivSprincip i de nyere Europæiske Nationallitteraturer . Af den Germanifie LenSforfatning , af Ridderstandens med Forrettigheder giengieldte Ryttertieneste og af begge under Indflydelsen af det sinere , hovifle Selskabsliv ( eoultnisie ) , af <b>Kvinderne</b> ( 8 » - lllnterie ) og af Geistligheden , fom nu sluttede sig til denne stedse mcrgtigere AandSretning , havde Nemlig den ideale Ridderaand ( enevalerie ) , hvis aandige Loftestcrnger var Horder , Kicrrlighed og Religion , og som havde vundet <b>Bevidsthed</b> under Korstogene , udviklet sig . Desaarsag maatte nu ogsaa Nationallitteraturerne , der paa samme Tid fremtraadte , faae Eharalteer , TendentS og

<h4>1919</h4>Farve fra Nationalfslelsen , men modificeret ved denne ridderlige TidSaand , og , jo mere det ene eller andet af disse Elementer havde Overvcrgten, antage en folkeligere eller kunstmcrSsigere Skikkelse . Derved opstod ved Siden af den nationale ogsaa en principiel Forfliellighed i den formelle <b>Dannelse</b> , og nu kunde ogsaa den forste , hvilende paa en bredere Grundvold , udville sig mere uindfircenket og fremtrcede skarpere markeret . Dette har ogsaa i Virke « ligheden saameget viist fig i den Syd « og Nordfranske Nationallitteratur i dette Tidsrum, at der da ikke kan vcrre Tale om en almindelig Franfi Litteraturhistorie , men lun om en speciel angaaende enhver af disse i Frankrig ved Siden af hinanden beftaaende Ssfterlitteraturer, hvoraf vel egentlig ikke den ene er celdre end den anden , men dog den Sydfranske forst maa komme i Betragtning , fordi den forst sil en tunstmcrSfig Behandling og derved har indvirket betydeligt paa den anden . Beboerne af det sydlige Frankrig , i Almindelighed betegnede ved det historist vage Navn Provene , aler , havde , begunstigede af en yderst frugtbar Jordbund og en scrrdeleS mild Himmel, tidligere end de fleste ovrige Beboere af det barbarifie Europa , allerede faaet en hoiere <b>Dannelse</b> fra den Fokcrifie ( Phonicifie ) Koloni Massilia. Denne <b>Dannelse</b> befcestedeS end mere af Romerne , der ligeledes var opdraget af Grcrkerne, saa at blomstrende HandelSstceder med Romerske Seeder og Love og Romersk Municipalforfatning endog havde overlevet nordisse Barbarers odelceggende Indfald og de Spanske Araberes odelceggende Streiftog . Her maatte deSaarsag forst de ny Scrder antage en mere ideal Retning ; her kunde snart igien fremstaae et sinere Selskabsliv ; men her havde ogsaa det mere dannede ridderlige Samfund adskilt sig skarpere fra Folket , her maatte ogsaa Poesien, Begyndelsen til al Nationallitteratur , forst <b>vise</b> sig som Kunstpoesi , og virkelig er den Sydfrcmfie Nationallitteratur eller Troubadourpoesien den crldste Nyeuropcrifie Kunstlyrik . Denne har vel ncermeft udviklet sig af den kunstige Latinfie Kirkepoesi , Hymnodien , iscrr i formel og musikalsk Retning ; men da dens fuldftcrndige <b>Uddannelse</b> faldt i RidderaandenS mest glimrende Periode , nemlig Slutningen af det 11 te og Begyndelsen af det 12 te Aarhundrede , og fortrinsvis udgik fra det adelige Samfund og de Stores Hoffer , under <b>Kvindernes</b> store Indflydelse , saa maatte den derved nodvendig uddanne fig i saadanue Hovedformer , som Canzone ( KicerlighedSsangen ) , SirventeS ( Wredigt ) og Tenzone ( Kampdigt ) . Fiernere Folger af Maaden , hvorpaa denne Poesi fremstod og uddannedes, var dens stedse tiltagende Adskillelse

<h4>1948</h4>Aand , Allegorien er meget grovere , i Stedet for den finere Ironi fremtroeder en Satire , der ei engang staaner det Helligste ; endog Spiren til de moderne kommunistiske Ideer om Kiodets Emancipation og FoelleSfiab i Eiendommen og <b>Kvinder</b> sindes der ; denne saakaldte Fortscettelse af hint celdre Digt fremkaldte isoer de deri angrebne Geistliges Fordsmmelscr og paa den anden Side de Lcerdes Forsvar og Udtydninger ved underskudte videnskabelige Tendentser og hemmelige Lcerdomme , og netop paa Grund af denne Kamp sil det Franske Publikum, der stedse mere hyldede hine realistiske Anskuelser , desto ftorre Smag for samme Skrift . Iblandt Udgaverne er den nyeste og bedste af Meon ( 4 Bd . , 1813 ) . Et Bevis paa , hvor yndet den poetiske Form er at ren videnskabelige og prosaifie Gienftande behandledes i Rim ; saaledes gives der rimede naturhistoriske Arbeider , Klostcrregler m . m . Mere poetisk end denne skolastiske <b>Visdom</b> udtaler sig den folkelige <b>Visdom</b> i ofte meget naiv ' kiernefulde Ordsprog , hvilke Digterne allerede tildels samlede , saasom hoverde » ou Onntes lle nyere Samlinger af denne Slags er ~ pil » verl » e » pnpulairet , " ( 1831 ) vg livre lle « prnverbe » " ( 1842 ) . Paa Grund af denne tidlige og rige Udvikling af den epifie og didattiste Poesi hos Nordfrsnfimcrndene er det naturligt , at Kunstlyrikken hoS dem uddannede sig langt senere og efter den Provencalske « Monster . DeSuden fremstod forst under Philip Augusts Efterfolgere, isoer under det <b>kvindelige</b> Regimen i c af Dronning Blanche , Grevinderne af Flandern og Champagne , en Hof-Kunftlyrilten betingende sinere Selskabelighed . Da derhos Franstmcendene ved den kongelige Stammes hyppige cegtefiabelige Forbindelser med sydlige Fyrstehuse , ved Korstogene og Albigensertrigene blev modtageligere for Sydens <b>Dannelse</b> og traadte i noermere Samkvem dermed , maatte nsdvendig Troubadourernes Poesi tiene som Monster for den Nordfranske i en Retning , som denne hidtil ncesten siet ikke tiendte . I Slutningen af det 12 te og iscer i den forfte Halvdel af det 13 de Aarhundrede blomstrede denne Poesi her , da den allerede i sit eget Hiem var kommet i Forfald . Konger og Fyrster , som Johan af Brienne , Thibaut den Fierde af Navarra , Henrik den Tredie af Brabant , endog den grumme Carl af Anjou , Konge af Neapel o . fi . kappedes med deres Hofdigtere ; ogsaa i Nordfrankrig blev Digtekunsten en adelig Vederkvægelse og Horte til en davoerende Hofmands ridderlige <b>Uddannelse</b> . HofsangerneS Tal tiltog betydeligt , og Laborde i fin „ Rsß « i sui I « musique " ncevner 136 Visedigtere i det 12 te og 13 de Aarhundrede . iblandt hvilke ogsaa fiere

<h4>1989</h4>saaledeS : „ den mindst lyriste af alle Mennesker i den mindst lyrifie af alle Tidsaldre . " Dog ftaaer Rousseau , hvad Sproget angaaer , hsit over sine Medbeilere . Romanerne , der i enhver Nationallitteratur spiller en vigtig Rolle og nceften altid kan tiene som Maaleftok for Dommen om den stsrre LceseverdcnS <b>Dannelse</b> og Smag , var i Ludvig den FiortendeS Tidsalder meget talrige, og Studiet af denne Gren afLitteraturen tillader Lceseren at giore dybe Indblik i Datidens <b>Aand</b> og Smag . Det er vcrrdt at lcegge Mcerke til , at det varede lcrnge , inden ClaSsiciSmen fandt Indgang i Romanlitteraturen , hvilken sidste udelukkende var yndet af det Publikum , som ikke fandt Behag i Digternes kolde Correcthed . Gautier de la Calprenede , dod 1663 , en Mand med en kiek Fantasi , men uden cefthetifi Besindighed , var den , der forst saaledeS bearbeidede Begivenheder af den Romerste og Grcrste Historie i den crldre Ridderromans Maneer , at kun Navnene forblev Grcrste og Romerske , men Eventyrene selv ganske faldt ind i den romantiske Riddertid . Denne Maneer fortsattes end mere af Madeleine deScudery; den « åndrige Grevinde Lafayette sirco ogsaa historiske Romaner . Smagen for de Spanske Noveller udbredtes ; men iscrr syntes Publikum om F ee « Eventyr . Charles Perrault , dsd 1703 , Fenelon , bersmt ved sin „ 1 V ' I « ! Nl ! < ilil : " , og Grevinden af Aulnoy, giordc Forssg i denne Retning . Ant . Galland firev en god Overscrttelse af „Tusend og een Nat " ; men iscrr har Grev Antony Hamilton , dsd 1720 , forhvervet sig et Navn ved fine Eventyr . Som Forfattere af komifie Romaner udmcertede sig Paul Scarron , 1598 — 1660 , og iscer Abain Lesage , 1668 — 1748 , efter Moliere den ypperste Skildrer af fin Tids Seeder . Kunsten at skrive elegante Breve blev mere og mere almindelig , og vi har i Samlingen af enhver udmcrrket Skribents Vcrrker ogsaa hanS Brevverling . I denne Henseende glimrer iscer de sire <b>Kvinder</b> : Babet , Boursault ' S aandrige Elskede , Marquisen af M a in ten on , Marquisen af Sevigne og ComteSsen af Staal , 1693 — 1750.

<h4>3166</h4>i det <b>kvindelige</b> Legeme ligefra Undfangelsen indtil Enden af FsdselenS umiddelbare Felger for <b>Moder</b> og Barn , men tillige angiver Midlerne , hvorved disse Foregaaenders regelmceSsige Gang befordres og dens Uregelmcessigheder paa en passende Maade forebygges. Da denne Videnskabs sidste Del , den praktiste , dog ogsaa ofte foreskriver umiddelbar virksom Hielp , hvortil atter udfordres en <b>vis</b> Fcerdighed , saa indbefatter Fsdselshielpen ogsaa enKunft , HorlsSningskunsten Opsnuste . t > iei3 ) , hvis Udsvelse er af saa stor Vigtighed for Mennefleflcegten , at Tilladelsen hertil i de fleste civiliserede Stater kun meddeles scerlig deri prsvede Lceger , saakaldte Fsdselshielpere eller Accoucheurs , medens den dicetetifie og therapeutiste Behandling af en Svanger , en Fedende eller Barselkvinde kan overlades til enhver anden Lcrge . naar intet manuelt Indgreb i Forholdet mellem <b>Moder</b> oq Barn udkrceves ; ved lette , regelmceSsige Fsdster kan Jordemoderen yde den fornedne Bistand . Man bsr derfor ikke ansee Fsdselshielpen som.en Del af Medicinen i indflrcenket Forstand eller af Chirurgien , da ei blot <b>Kundskab</b> til begge disse Afdelinger maa vcere forenede , men ogsaa Meget, som hine i dereS scedvcmlige Betydning ikke omfatter , maa vcere tilstede for at danne en fuldkommen FsbselShielper . Af denne Grund udkrceveS der til at lcere Fsdselshielpen en scerlig Klinik , hvori den til FedselShielpen henhsrende Pathologi og Therapi lceres , og hvortil den medicinske og chirurgifie Klinik tiener som Forberedelser . Forsvelserne til Fsdselshielper- Operationer foretages paa et Ph ant om ( s . s . ) . Saadanne Operationer bliver nsdvendige , naar en Paafiyndelse af Fsdselen udfordres mod Svaghed , Afthma , Blsdninger eller andre enten allerede indtraadte eller dog befrygtede Ildebefindender , hvilke gisr Moderen FsdselSanstrengelsernes Fortscettelse umulig eller dog meget farlig ; eller naar Frugtens Udgang forhindres ved dens egen Stsrrelse eller ved BcetkenetS Smalhed ; eller naar Barnets Leie modscetter sig Giennemgangen igienneM Fedselsdelene; eller naar Uregelmæssigheder i de saavel Moderen som Barnet tilhsrende Dele truer Een af Begge eller Begge tilsammen med Fare , f . Er . tykke Wggehinder , en for kort eller for lang Navlestreng , sammes Slyngning, Knude , Fremfald , Ssnderrivning o . s . v. FedselShielpen « Historie stutter sig ncer til hele LcegevidenfiabenS ; kun stod FedselShielpen indtil det 18 de Aarhundrede i sin <b>Uddannelse</b> stedse tilbage for Medicinens evrige Dele , da den maatte kcempe med langt flere Fordomme , end hin . Allerede i de celdste historiske Skrifter ncevneS lordemedrene som en sccrstilt Klasse , og saavel hos Grcekerne som hos Romerne « rede

<h4>4535</h4>ung Alder udmoerkede G . sig ved usoedvanlige SnilletS Gaver . Ni Aar gammel firev han VerS og blev i fit 15 de Aar Doctor i Lovkyndigheden. Aaret efter fulgte han med. O lden Barneveldt ( s . s . ) , der som Sendemand gik til Paris , og vakte her ved sin <b>Loerdom</b> og <b>Aand</b> Kong Henrik den FierdeS Opmoerksomhed. I en Alder af sytten Aar optraadte han som Sagforer . Hans Hukommelse var saa ftoerk , at han efter Msnstringen af et Regiment fial have kunnet opregne Soldaternes Navne . Han havde ogsaa ualmindeligt Kiendfiab til Theologi , Historie , Retsvidenskab og Poesi ; dertil en sieldeu Evne til med Smag og Klarhed at udtrykke sine Tanker paa ModerSmaalet, Franst og Latin , endvidere en utroettelig Arbejdsomhed , rene Soeder og eN for alt LEdelt og Godt aaben Sands . Med disse Egenfiaber steg han stedse heiere i Anseelse og blev 1607 Generalsifial og 1613 Raadspensionair i Rotterdam . Paa den Tid foruroligedes den Hollandfie Fristat ved Tvistigheder imellem to Religion Spartier , Arminianere og Gomarifter . Prinds Moritz af Nassau , der tragtede efter en udvidet Magt , sluttede sig med sine Tilhoengere til de sidste ; til de fsrste Horte Olden Barneveldt , og G . underststtede dem ved sine Skrifter og sit ansete Navn . Den snu PrindS Moritz benyttede denne Lejlighed til at stille sig af med fine Modstandere , Republikanerne ; den gamle cervoerdige Olden Barneveldt blev henrettet , og G . dsmt til livsvarigt Fcengsel paa Slottet Lsvenftein . Derfra blev han befriet af fin cedle <b>Hustru</b> . Hun forte til Focngstet en Kiste fuld af Boger ; da disse var udtaget , bad hun sin Mand at lcegge sig i Kisten og viste ham , at hun i Siderne havde boret flere Huller , for at han itte stulde kvoeleS derinde . G . fulgte hendes Raad og blev frelst . Hustruen foregav , at han var syg , blev derfor i Foengstet og lod , som om hun var sysselsat med at pleie ham . Da hun endelig med Sikkerhed troede ham reddet , viste hun Fangevogterne den tomme Seng og fortalte dem , hvad der var fieet . Det Oranifie Parti blev forbittret , og Nogle vilde indespcrrre hende istedetfor Manden ; men Flertallet frikiendte den snilde <b>Kvinde</b> , og med megen Berommelse begav hun sig til Belgien , hvor hendes sEgtefcrlle da var . Derfra flyttede de til Paris ; her blev G . vel modtaget og sil ved RichelieuS Indflydelse aarlig 1000 Daler at leve af . Det i Holland herskende Parti sogte at opirre Kongen og Richelieu imod ham ; men forgiceveS , og han hevnede sig ved med Iver at arbeide for sit utaknemmelige FoedrelandS Bedste . Isvrigt opholdt han sig omtrent 11 Aar i Paris og begyndte pu ved fine Skrifter at indtage en as de horderligste

<h4>4793</h4>endelig tlllcempede Ro omfattede den lynfie Poesi ogsaa Livets Glceder og de deraf fremkaldte Fslelser , hvorhos hin sine Sands og sine stedse tydelig udtalte sig , hvorved Livet sik sterre Inde , Nydelsen forcedledeS og Fremstillingerne deraf sit en eiendommelig Inde , ligesom de ogsaa udmcerlede sig ved Naivitet . Iblandt dem , der herved som ogsaa ved <b>Uddannelsen</b> af Musikken og Opfindelsen af forfliellige Former for denne lyriske Poesi har udmcerlet sig , maa iseer ncevneS ArchilochuS ( s . s . ) fra ParoS , Opfinder af Jamben; TyrtceuS ( s . s . ) fra Milet , Digter af Krigssange ; Kal linu S ( s . s . ) , fra EphesuS , Opfinder af det elegifie Stavelsemaal ; Altman ( s . s . ) , en Lyder , A ri on , fra Methymno , som uddannede Dithyramben ; Terpander ( s . s ) , fra AntiSsa , Opfinder af den syvstrengede Lyra ; den omme S appho ( s . s . ) fra Mitylene ; hendes Landsmand AlcceuS lf s . ) og beggeS Samtidige Erinna ( s . s ) ; MimnermuS ( s . s . ) fra Kolophon , Floitespiller; StesichoruS fra Himera ; IbykuS fra Rhegium ; Analreon ( s . s . ) og Simonides ( s . s . ) fra AmorgoS ; BacchylideS ( s , s . ) fra KeoS ; Hipponar fra EphesuS ; Pindar ( s.s . ) og Korinna ( s . s . ) fra Tanagra, Pindar ' S Veninde og <b>Lcererinde</b> . Som gnomifie Digtere ncevneS So lon ( f . f ) , TheogniS ( s . s ) , SimonideS ( f . s . ) fra KeoS , PythagoraS ( s.s . ) ogXenophaneS ( s . s . ) , fra Kolophon ; som Fabeldigter A3sopuS ( f . s . ) . Betragter man denne Tidsalders Filosofi , saa sinder man den iscer henvendt paa det Praktiske , fordi Alt udgik herfra og henvifte hertil . Fra dette Synspunkt maa man betragte Grækenlands saakaldte syv <b>Vise</b> , Periander ( s.s . ) , PittakuS ( s , s . ) , Thales ( s . f . ) , Solon ( f . s . ) , Bia « ( f . Chilon og Kleobulus . DereS Tankesprog er Leveregler , fremkaldte ved praktiske Forholde , og ofte tun Udtrykket af den sieblikkelige Fslelse. Men fordi Viden dog er Grundvolden for den sande <b>Visdom</b> , saa maatte man ved videre Granskning k » mme til denne Viden , og saaledeS udgit den theoretifie Philosophi ilke aldeles tom heraf . Thales blev Stifter af den lonisse Philosophi , og fra denne Tid af Horte Poesien op at vcere Summen af alt Videvcerdigt, den eneste <b>Lcererinde</b> og Opdragerinoe . Hidtil var den tillige traadt i Stedet for Historie, Philosophi og Religion ; hvad man vilde overlevere til Efterverdenen , hvad man vilde meddele af Levevisdom og <b>Kundskaber</b> , hvad man havde at meddele af religioS Undervisning, udtryktes i VerS . Dette fiulde for Fremtiden blive anderledes ; Statsborgerens Liv maatte ogsaa have en betydelig Indflydelse paa Sproget , » g de offentlig « Forhandlinger , i

<h4>4948</h4>traadte G . paany i StatSstyrelsen , idet han i Forening med Mol < s dannede et nyt Ministerium, og selv overtog Portefeuillen for den offentlige Undervisning . Da dette Ministerium blev modificeret den 15 de April 1837 , traadte G . tilligemed de svrige Medlemmer af det doctrinaire Parti tilbage . Senere indtraadte han i de 221 ' S Coalition , som vilde styrte Mole , og da i Januar 1839 Kamrene blev oploste , blev han gienvalgt i sin Valgkreds . Den 28 de Oktober 1840 overtog han Portefeuillen som Udenrigsminister i det ny Ministerium, hvis <b>Dannelse</b> isoer var hanS Vcrrk . Skisndt han var i hsi Grad upopulair , dels vaa Grund af at man tillagde ham den Hensigt , at ville stutte et Forbund med England, hvilket hoS Mcrngden skaffede ham Navnet: ministre 6 e I ' etr « n ^ e » , delS ogsaa formedelst en <b>vis</b> Stivsindethed HoS ham , holdt han sig dog vaa sin Post indtil endelig Februar- Revolutionen iParis ( lß4B ) nodte ham tilligemed de svrige Ministre til at fiygte . Der blev nu af Staten anlagt Sag mod Ministrene , som imidlertid blev frikiendt , hvorpaa de fleste af dem vendte tilbage , medens G . endnu i loengere Tid vedblev at opholde sig i England , hvortil han var flygtet , og hvorfra han forst senere vendte tilbage . Siden den Tid har han trukket sig tilbage fra det offentlige politiske Liv , og paany offret sine Krcrfter til litteraire Arbeider . Han har saaledeS dels bessrget nye Udgaver og Fortscettelser af adskillige af HanS tidligere Vanker , f . Er . : „Histoire « les llu Znuveinement revre . sentnti ! en Hurnpi , ( 2 Bd . PariS 1851 ) ; ~ Hiswire de 111 revolution ete . " ( 2 Bd . Paris 1850 ) ; dels leveret fiere nye Arbeider f Er . : et etudes morsles" ( 1 Bd . PariS 1852 ) ; „ Bn3llßveare et son temns " ( PariS 1852 ) ; et son temvs " ( PariS 1852 ) o . fi . HanS litteroere og videnstabelige Arbeider have vundet almindelig Anerkiendelsc ; navnlig har han store Fortienester af Udbredelsen af de historiske Studier i Frankrig , dels ved at grundlcegge komites og ved at befordre Udgivelsen af de vigtigste Kildeskrifter , dels ogsaa ved HanS egne Skrifter og ForelceSninger . HanS Bearbeidelse af et ecrits lle ( 4 Bd . , PariS 1840 ) , som han ledsagede med en biographist Indledning, skaffede ham den Asre , at hans Billede blev ovhoengt i SeSsionSsalen i Washington . Hans fsrste <b>Kone</b> , Elisabeth Charlotte Pauline, f . de Meulan , fedt i PariS 1773 , gift med G . 1812 og dsd 1827 , er bekiendt som Forfatterinde til fiere fortrinlige Skrifter om Opdragelsen . Blandt dem kan neevneS eullwts " ( 2 Bd . , Pari « 1812 ) ; „ I / ecolier on

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112200023'target=_>Hofacker, Ludwig, 1857, Prædikener paa alle Søn- og Helligdage tilligemed andre Taler</a><h4>1489</h4>Band til Biin o . f . v. Videre end til saadanne ydre Kjendsgjerninger naae ikke lettelig de Fortellinger , som gaae i Folkemunden. Men dermed kunde den kanancriste <b>Kvinde</b> ikke sinde sig tilfredsstillet ; hun forhorte sig videre ; hun horte , at han ncdstammede fra Davids Slcrgt , og maaste var den forjcrttede Davids Son ; hun sogte Underretning om hans Dpfsrsel , om hans Hjertelag, og horte om hans Kjcrrlighed til de Elendige , om hans store Barmhjertighet Og hvad hun saaledes horte om ham , det frembragte hos hende en übegraendset Hoiagtelse for Jesus og en urokkelig Tiltro til hans Hjerte . O , mine kjoere Tilhorere ! Hvad stulle vi sige om denne hedenske <b>Kvinde</b> , som faaledes aunammede og benyttede de ringe og upaalidelige Etterretninger , hun havde om Jesus , at den staerke Tro , som Frelseren selv beundrede hos hende , kunde vore frem af dette lille Frokorn ? Hvad stulle vi sige om denne Kjcerlighed til Sandheden , om denne Trostab i at bevare , hvad hun havde modtaget? Sandelig ! Denne <b>Kvinde</b> er sat som et lysende og fordsmmende Tegn for os og denne Tidsalders Mennesker . Mange Hedninger , som dog ikke kjende Guds Vei , <b>vise</b> for ncrrvcerende den stsrste Begjcrrlighed ester Guds Ord og ester Loerere . Folk , som forst for kort Tid siden have lcrrt Evangeliet at kjende , lcere med den stsrste Begjerlighed at loese , for at kunne lcese Guds dyrebare Ord ; de agte Guds Ord for sin dyrebareste Skat ; de kunne ikke blive mcrtte af at hsre om sin Jesus ; de soge Frelseren ; de soge hans Ansigt ; med mange Bonner og Taarer soge de Guds Naade og sine Synders Forladelse i Lammets Blod . Herrens Herlighed opgaaer over mange Hedninger , og vi ste dem vandre i dens Lys . 33 i derimod have Guds Ord i saa rigeligt <b>Maal</b> ; vi have det hjemme og i Kirken , og agte denne dyrebare Skat som noget Almindeligt , og det forekommer os , at det saa maa verre . Sandheden , det evige Evangelium have vi i saa rigeligt <b>Maal</b> , og vi rore os ikke , og de Fleste blive i sine Synder, i sit Liv ester Fcedrenes Viis , i sin aandelige Uforstand og Blindhed , og takke ikke for Guds « fortjente Forbarmelse , og tcrnke ikke paa at takke . Hvad vil dog Gud maatte foretage sig med os ! Naar han lader sit dyrebare Evangelium prcrdikes for os ; naar han lader Christum afmale for vore Vine ; naar han lader et saa rigeligt besat Bord fremscrtte for os , og vi ikke aabne Munden for at nyde , men for en Deel endog vcemmes ved denne Spise , medens mange Tusinder vilde vare glade ved de Brodfmuler, som falde fra vort Bord ; hvad vil da Gud blive nsdt til at foretage med sin dovne , kolde , mattte , stolte Christenhed ? Det vil han maatte foretage med den , at han aldeles omstoder dens Lysestage , hvortil der i vor Tid allerede er Udsigter , saa at

<h4>2859</h4>Dette er da det jordiske Sind , disse ere de elendige Fslger , dette er deres Skjaebne , som hcenge sit Hjerte ved det Timelige og Forgjengelige , enten det nu er timelige Goder , eller Glaeder , eller Hceder . Men ikke meget bedre ere 2. De , som ville dele sig mellem Verden og Gud , de som have et deelt Hjerte . Man har Indtryk af Gud og hans Sandhed , og vil dog ikke forlade Verden : man har vel nu og da erkjendt Evangeliets Kraft ; fslt , at det er godt at overgive sig til Herren og handle ester hans Villie ; men Gjenfsdelsens Scrd , Guds Ord , er faldet blandt Torne , og ved Siden af Larngslen efter at blive sa ' ! ig haver sig mangeslags Tragten efter Rigdom , 3 Ere , Magelighed , et sorgfrit , behageligt Liv , som man med al Magt snfler at forbinde med sin Tragten efter Himmelen . Nu straeoer man da paa alle mulige Maader at give sine urene Omsorger , sine usle Begjæringer et crrbart , christcligt , pligtmcrssigt Anstrsg , stjuler sin hemmelige Gjerrighcd under det Kald , man har at ssrge for <b>Kone</b> og Born eller for det almindelige Bedste ; sin forfengelige Adspredelsessyge under den foregivne Pligt , at leve overeensstemmende med sine Forholde , at omgaaes sine Medmennesker paa en venlig og forekommende Maade overensstemmende med den almindelige Menneffekjcrrlighed , og man tilfoier , at man ved at drage sig tilbage fra saadanne tilladte Fornsielser kan stade sit Rygte , ja vel endog give Forargelse ; sin Mrgjerrighed tildcrkker man med et Figenblad , som man kalder <b>Uddannelse</b> af sine Evner , Gavnelyst og Iver for det almene Bcdste ! — Man har vel Noget , som ligner Gudsfrygt , men det er ikke den rette , men en Traeldomsftygt, som kommer af , at man har en ond Samvittighed for sin Oienstalkhed ; man hsrer vel gjerne noget Godt , men det faester sig ikke ; det fslges ikke af noget levende Bifald , nogen Tilvcrxt i Lys , Liv og Erfaring ; man forglemmer det hurtig igjen , og naar man en Stund har varet from med de Fromme , lever man igjen verdsligt med denne Verdens Born ; man har vel nogen Lcrngscl efter <b>Sjelens</b> Frihed og Salighed , men man vover Intet derfor ; frygtsom regner man efter , hvormeget man maa give Slip paa , tabe , forncegte og taale , hvis man beslutter for fuldt Alvor at overgive sig til Herren ; man stjoelder sig selv ud for en Synder , og tåger ogsaa et Tillsb til Omvendelse , men man hader ikke Synden , og klager ofte over indre Fiender , med hvilke man dog staaer i det bedste Venstab , dersom man vil ste neie til og aflagge en redelig Bekjendelse , ja hvilke man btvarer som sin Oiesteen . Man vilde gjerne tilhsre Christo , men man kommer dog ikke til ham , fordi man bestandig har saa mange andre Sager at betcrnke og ftrge

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013072208224'target=_>, 1880, Vidnesbyrd fra den norske Kirke</a><h4>431</h4>Kobber , Solo og Guld og blev til et stort Bjerg , der opfyldte al Jorden ( Dan . 2 ) . Det er Jesu Kristi Rige , der ved Naadens Midler i Ord og Sakramenter gaar sin seierrige Gang gjennem Verden og lader sig ikke standse ; thi Faderen har sagt til Sonnen : „ Jeg vil give dig Hedningerne til Arv og Verdens Ender til din Giendom ( Ps . 2 , 8 ) . Derfor vender vi i den ncervcerende Tid , da Vantro og Partiaaud kappes om at saa Ugudeligheds og Splittelsens Udsced i vore hjemlige Menigheders Jordbund , med Gloede og Haab vort Blik til Menighedens Arbeide paa Missionsmarkerne og fryder os ved at se Zions Mure byggede derude , en Brud , der smykkes og beredes for den himmelske Brudgom . Se dog , hvor Missionen lofter Kristi Riges Banner blandt Folkene i den nerverende Tid ! Det umaadelige Kina har lukket Dsrene op for Evangeliets Forkyndelfe ; paa Japans Ver fpredes Kristi Evangelium i Skrift og i Tale . I det folkerige Indien lyder Evangeliets Indbydelfe lige ved Siden af Afgudstemplernes Begge , Missionens Skoler undergraver Hedenskabet Forstandsninger ; <b>Kvinder</b> , der har ofret sit Liv til Herrens Tjeneste , baner sig Vei til de hinduiske Hustruers og Modres indelukkede Kamre . Paa Afrikas Ostkyst fynges der nu til Herrens Pris , hvor man for fire Aar siden endnu kun Horte Slavehandlernes vilde Rllab ; et kristeligt Gudshus hever sit Spir midt paa Zanzibars tidligere Slavetorv , og fra Kysten drager allerede hele Skarer af Missionerer til Afrikas nylig übekjendte indre . Paa Sydhavsoerne har Evangeliet forvandlet tidligere Menneskeædere til <b>aandelig</b> fattige , sagtmodige og fredfommelige Kristi Difciple . I Australiens Skove har Missionererne banet sig Vei med Oxen i Haand for at faa indbyde deres raa Beboere til Kristi Nadvere . Paa Sydafrikas Bakker hores Krig og Rygte om Krig ; men vi er forvisfede om , at Herrens Tugtelfer stal tjene til at aabne Arene for Herrens Rost . Og er der ikke i vor Kirkes Mission paa Madagaskar steet stsrre Ting , end nogen af os vovede at haabe for ti Aar siden ? I vor derverende Missions Skoler horer Tusinder as Born Ordet om ham , der har sagt : Lader de smaa komme til mig . Og af de voxne rives ikke faa faa udaf de Afgudsdyrkelsens eller Hykleriets Snarer , der omgiver dem . Endog saadanne , der er blevne meget gamle i Hedenskabets Forvildelser , sporger om Veien til Livet . Og nåar det ster , Venner , da ster et overvettes stort Under . En af vore Missionerer skriver blandt andet om en Mand paa mellem 80 og 90 Aar , som han nylig har dobt : „ Hans <b>Kundskaber</b> er ikke store ; men at han er en fortabt Synder , og at Jesus er Synderes Frelser , staar saa klart for ham fom for nogen i Menigheden her . Han gleder sig

<h4>511</h4>den fsrste Altergang efter Bryllupet . Det er en vakker Skik for Kristne , der vil fore sin LEgtestand i Herren ; hvor trcrnger de dog at ssge al den Kraft og Naade dertil , som Herrens Naademidler har at give dem ; ja , stal ægtestanden virkelig blive fort i Herren , da maa de ikke blot hin forste Gang soge Kirken og Sakramentet , nei , de maa atter og atter soge didhen ; Kirkestien maa altid vcrre dem en kjendt og kjcer Sti ; mellem deres Hus og Kirkehuset maa der sinde en levende forbindelse Sted ; Samfundet med Herren og hans Menighed er det jo , ved hvilket Naadens og Livets Stromme tilflyder os herovenfra . Lever LEgteparret faaledes i inderlig Forbindelse med Herren og hans Menighed , da vil de ogsaa faa <b>Visdom</b> og Naade til at indrette sit Husliv saaledes , at deres Hus bliver et Gudshus i det Smaa , en Kirke , hvori Husfaderen er Prest , <b>Husmoderen</b> deri som i alt Andet hans trofaste Medhjcrlp , medens Bornene og Tyendet er Menigheden . Og Herren , den uelfignede Frelser , bor selv i Huset , gaar ud og ind der og aabenbarer sin Herlighed . Som en ret Prest vil da Husfaderen flittig bede baade for og med sine Husfolk , lcrse Guds Ord for dem og give dem flittig Anledning til at hore Guds Ord ; eftersom Leiligheden kalder ham , vil han ogsaa advare og formane , unde og veilede , viser tugte og troste hver enkelt af dem med Guds Ord og faaledes holde kristelig Stik og Orden i Huset . Ak , at daglig Husandagt dog engang maatte blive en fast Skik i vore Huse ! Naar Husfaderen faaledes i Herrens Kraft og <b>Aand</b> forvalter fit prestelige Embede i Huset , da vil han ogsaa staa for sine Husfolk ifsrt den rette prestelige Vcerdighed, saa at selv de stridige Gemytter boier sig for ham og adlyder . Saa kan Huslivet fores i Fred og Endrægtighed , uden Strid og Splid , Trcette og Tvedragt , og gaar saa ogsaa Hustruen ved stn Mands Side som hans trofaste Medhjcelp i alle Ting , da vil det foles af Alle , at der er godt at vcere ; der hersker en liflig Hygge , som hverken Verdens Forfængelighed og Rigdom eller Verdens raa Vellyst kan give . Vistnok gaar det heller ikke i et saadant Hus af uden megen Skrobelighed , mange Fald , mange Kampe , Trcengsler og Modgange ; det horer nu engang med til Kristenlivet her i denne Jammerdal ; men det mcrrkes dog ogsaa , at Herren er tilstede , som forvandler Vand til Vin ; han ovreiser af Faldet og lceger han giver Seier i Kampen og Trost i Sorgen ; han hjcelper at bcrre Korset oq forvandler dets Bitterhed til idel Todhed ved de salige Frugter , han lader sremvore deraf . Netop Skrobelighederne , Kampene og Trcengflerne , som Huslivet bringer med sig , noder og driver et kristent LEgtepar til at holde sig end inderligere til lesum for hos

<h4>854</h4>evige Skatte vente kun paa fattige Synderhjerter , som ville modtage dem og lade sig delagtiggjsre i de evige Rigdomme , som Me Mol og Rust kunne forteere og ikke Tyvehaand gjennembryde og stjcele , ingen jordisse Forholde , ingen Nod og Dod og Djcrvel berove os , nemlig Syndernes Forladelse , Fred og Glnde i den Helligaand. Hvor finde vi da disse os saa dyrt forhvervede Skatte ? Hvorledes bringes os disse vor Immanuels Gaver ? I Ordet og . Sakramenterne . Deri som ved de rette Midler ssjenkes den Hellig / aand med al oans Naade og Gave . Men her gjcelder det , at du ogsaa bruger Midlerne ret . Det se vi her . Ordet har lydt med Myndighed . Ordet om Synden i Menneskelivet og om Naaden i Kristo Jesu har Herren netop fremholdt . Og saa udbryder en <b>Kvinde</b> af Folket i Lovprisning . Ja , love og prise burde det ogsaa os , dog ikke som <b>Kvinden</b> Moderen , men Sonnen . Ak hvor dog dette arme Hjerte er en forvendt Ting ! Alt Andet kan det hcrnge sig ved , elske og forlade sig paa , grunde sin Lykke og Salighed paa : men trods man har Ham i Ordet , og Hans ssjonne Skikkelse deri males En for Ojnene Gang efter Gang , horer man Ham ikke , fer Ham ikke , faar Ham ikle kjcrr . Ugudelige og Ryggeslofe , Religiofe og Fromme Hans Hellighed og Renhed i Loven forfcerder dem ikke ; Hans Kjcerlighed og Barmhjertighed i Evangelium overtaler dem ikke til at lade sig forlige med Gud . Verdenstjcerlighed og Egenretfcerdighed hjcelpe sig hver paa sin Vei Ham foruden . Tet spredte Sind og den forfengelige <b>Aand</b> overser den ydmyge og sagtmodige Frelserfkiktelse . Kun fattige Syndere , som i Folelsen af sin dybe Jammer maa have en Frelser , hore Hans Rost og annamme og bevare den , saa den lig en levende Seed faar under den Helligaands Velsignelse feste Rod i Hjertet og spire i Veiret og beere Herren modne Frugter , ogsaa den Frugt at blive ret eedrue og aarvaagne til Bonnen , ransage daglig i Skrifterne og derigjennem lcere at vandre i Lyset og snart ved lesum Kristum under Troens gode Strid mod Satan , Verden og Kjodet naa <b>Maalet</b> ,

<h4>1809</h4>som I lcrgge saadan Vcrgt paa , alt delte bringer eder ingen Ros eller LEre hos Ham , som stal domme eder paa den yderste Dag . „ Thi intet Kjsd kan blive retfcrrdigt for Gud ved Lovens Gjerninger " . Men „ Kristus er bleven os , som <b>Visdom</b> fra Gud , saa og Retfærdighed og Helliggjorelfe og Forlosning " . Og kun den torstige , der vil tage Livsens Vand uforskyldt , som en fri , fuld Gave , kan saa sin Torst slukket ; men den , som kommer frem til Gud med nogensomhelst Betaling og vil hobe hos Ham , han faar intet , thi Gud er altfor stor til . at Han skulde have noget at fcrlge . Men , Ven , Han har megen Naade at give og er saare rund til at forlade . Jesu Kristi , Guds Sons Blod renser fra al Synd . O , Venner , de falske Profeter fore os vild , derfom vi lade os lede af dem ; de <b>vise</b> os bort fra den eneste Vej , som forer til Livet , Budens og Troens Vej , og ville , at vi paa en eller anden Maade stulle foge vor Frelse ios selv . Herren ser det og raaber til os : „ Vogter eder for de falske Profeter ! " Men de ere ej lette at adstille fra Herrens sande Profeter , hvem det bor os at lytte til . De bcrre intet Kains-Mcrrke i fin Pande , komme ikke til os fra den store Hob , som ved al Ugudeligheds Bedrift klart nok lcrgge for Dagen , hvilken Aands Born de ere . De falske Profeter ere at soge indenfor Kirkens egne Mure , og de foregive ikke blot , at de optrcede i Guds Navn , fom Hans Udsendinge , men soge ogsaa ved hele sin Fremtråden i Ord og i Gjerning at bevise sin guddommelige Sendelse. Ligesom deres Aandsbroder og Forbillede , Bileam , ere ogsaa de meget crngstelige for at komme i aabenbar Strid med Gud og Hans Menighed . Det Tungemaal , de tale , fynes ej at vcere forstjelligt fra det , som <b>moder</b> os i Guds hellige Ord og gjennem Hans sande Tjeneres Mund . Ja , ligesom vi fra hin det gamle Testamentes Lognprofet have en af de herligste Forjcrttelser om Kristus og en af de stjonneste Lovprisninger over Guds Borns salige Kaar ( 4 Mos . Kap . som findes i den hele Bibel , saa har Sandheden i de falske Profeter ofte endnu meget veltalende Tunger . Og , fom Herrens folgende Bestridelse af dem viser os , vide de ej blot at tale meget om Herren , men de kunne ogsaa fore falvelfesfulde Bonner i fin Mund ; ja der findes endog faadanne imellem dem , som boere i sig den tilkommende Verdens Krcefter og paa Dommens Dag ville kunne minde Herren om , at de ej alene have profeteret i Hans Navn , men endog have uddrevet Djcevle og gjort mange kraftige Gjerninger i Hans Navn . Og Faren , som <b>moder</b> os fra de falske Profeter , bliver tilvisse ikke mindre derved , at de slet ikke altid ere sig selv noget Falskneri bevidste . De ere ofte ikke blot Forfsrere,

<h4>1886</h4>Loerdom og tor Bogstav-Visdom kan uddrive de onde Aander og holde Nattens Skygger borte ? Eller kunne vi haabe , at den verdslige Oplysning og <b>Dannelse</b> , den fra Kristendommen lssrevne Folke-Oplysning kan sprede Morket ? Intet af dette . Morkets Fyrste , der saa vel kjender Mennesket og dets Svagheder , frygter hverken „ den falsteligen saakaldte <b>Kundskab</b> " og dens Profeter , ei heller Laodicceifl Lunkenhed og verdsligsindede Prcedikanter , ei heller ufri Lovtroelle og aaudelig forkroblede Bekjeudere , ei heller vakte Pralere , der ikke have faaet et nyt Hjerte . Han frygter kun lesum Kristum , det trofaste og sanddrue Vidne , han frygter den hellige Taare og Kærlighedens hellige Alvor , han frygter derfor ogsaa det sande , livskraftige Vidnesbyrd herom i Lcere og Liv . Vi loese i Aftostl . Gj . Cap . 19 , at syv Sonner af en jodifk Ippersteprcrst vilde besvcerge de onde Aander ved at ncevne Jesu og Pauli Navn , men den onde <b>Aand</b> svarede og sagde : lesum kjender jeg og Paulus ved jeg af ; men I , hvo ere . I ? og de bleve overvældede af den onde <b>Aand</b> og maatte flygte nogne og faarede . Saa er det og nu . Den onde <b>Aand</b> , som raader i Tiden , frygter kun lesum Kristum og ham korsfæstet og viger kun for Herrens fande Disciple , som i Pauli <b>Aand</b> kunne sige og i Livet bekjende : „ Jeg agter alt for Tab mod Ivperligheden af <b>Kundskaben</b> om Kristo Jesu min Herre " . ) Hvad tjener derfor til vor Fred , hvad kan afvcerge endnu for en Tid Straffedommen fra vort Folk ? Den fande , frigjorte , evangeliske Gudsfrygt , som beviser sit guddommelige Udspring ved Lydighed uuder Ordet . Vi maa bekjende den levende personlige Kristus og i vor Vandel i alle Retninger vidne om , at Kristendom erLcrgedom , og at Kristenliv er vel et Liv i Forsagelse , men ogsaa i daadskraftig Gjerning , i Kamp mod Morkets Magter for Hjem og Arne , for Fædreland og Konge , I Hjemmet , i Folkeskolen , i Hoistolen , i Retten , i Storthinget , i Kongens Raad vil den Herre Kristus vcere den raadende og hans Villie vor Lov ! Held os , om saa kan ske , held os , om flere og flere tor troede i Skranken for den Herre Kristus og i Pauli <b>Aand</b> vidne om ham , held os , om ogsaa i den dannede og hoiere stillede Del af vort Folk Mcend og <b>Kvinder</b> kunne fremstaa , som kunde fore Folkeaanden ind paa sit eneste rette Spor : Frigjorelsen i Kristo . Intet er sorgeligere end dette , at det har lykkets Satan at indbilde dannede Folk , at den sande Kristendom er 6 t med Pietisme og <b>aandelig</b> Sygelighed og at paa den anden Side Frihed for Kjodet er Tegn paa <b>aandelig</b> Frigjorelse . Maatte Herren i sin Naade lade den sande

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014112108104'target=_>, 1871, Nordisk Conversationslexicon</a><h4>120</h4>raderede Blade . Han stak et Martyrologium med 392 Helgenbilleder ; krigfsrende Fyrster travede hans Narvarelfe ved Beleiringer for at forevige deres Minde ved Radeernaalen . Dog undslog han sig for at afbilde sin egen Fsdebyes Beleiring af Franskmandene og lod sig ikke friste til at trade i fransk Tjeneste , men da Nancy blev indlemmet i Frankrig , besluttede han sig til at udvandre med sin <b>Hustru</b> til Floreuts . Han dsde imidlertid forinden , 1635. C.s Malerier ere mindre betydelige , men hans Raderinger , blandt hvilke ifar hans to Rakker » UiB6reB < le la gueire « ere bersmte , <b>vise</b> ham som en hsist aandrig Kunstner , begavet med skarp lagttagelse , megen Humor og Lune , men ogsaa med en spottende Ironi og et Hang til det Damoniste og Salsomme . Han har udfsrt c . 1400 Raderinger , deriblandt ogsaa Scener af den italienske Mastekomedie , af Festoptog, Optrin af Noveller ofv . Callus , s . Beenbrud . Callofitet , en Forhardelse af et ellers blsdt Vav , s . Ex . Huden ( „ haard Hud " ) eller af Saarrande , der da siges at blive callvse , osv. Calmar , Stapelstad paa Sverrigs sftlige Kyst ved det efter Byen benaunte ( Z sv . M . brede ) Sund , som adstiller Oland fra det faste Land . Staden omtales allerede af aldre arabiske Geographer; den var i Middelalderen og Tiden narmest efter Unionens Oplosning en af Sverrigs betydeligste Handelsstader og tillige den starkeste Grandsefastning mod Syd . C . var derfor i Krigene mellem Suerrig og Danmark bestandig et <b>Maal</b> for de forskjellige Partiers Angreb , alt efterfom den var i det enes eller det andets Hander . Ligeledes var den etsadvanligt Modested ved Fredsunderhandlingerne mellem begge Rigerne ; faadanne Msder holdtes her 1438 , 1472 , 1476 og 1483 ( Calmarrecessen ) . Christian IV . gik iland ved C . og erobrede Staden 1611 ( f . Calmarkrigen ) . Fsr 1647 , da den blev sdelagt ved en Ildebrand , laae den paa Fastlandet , men flyttedes derpaa til sin nuvarende Plads paa den tat ved Kysten liggende og dermed ved en Bro forbundne V Quarnholmen. C . har en god Havn , et Skibsvarft og fiere Fabriker ; Udfsrselen fornemmelig af Skov- Produkter er betydelig ; Tallet af Skibe , bestemte til udenlandsk Ssfart , belsb sig 1867 til 69 ( deraf 2 Dampfartsier ) paa 3090 Nylafter . Toloopvebsrfelen belsb sig s . A . til 193,974 Rd . Rigsm . Den har en Ssassuranceforening , en hsiere Elementarflole, flere Bankindretninger og Lcinslazareth. 9,421 I . ( 1868 ) . - Paa Fastlandet , strar s . for Forstaden , den saakaldte „ gamla Staden", ligger Calmarslot , bersmt ved den 1397 gjorte Aftale , som siden lagdes til Grund for de tre Rigers Forening ( f . Calmarunionen ) . Slottet forstjonnedes og udvidedes af Erik XIV . , der her holdt sit Hof fom Kronprinds , og endnu mere af Johan 111. , fom ofte opholdt sig her , men det forfaldt i det 18 deAarh . , brugtes en Tid somßranderi for Statens Regning , senere som Magasin og Fang,sel; fsrst i den nyeste Tid har man begyndt lidt efter lidt at restaurere det . Calmailan , indeholdende et Fladerum as sv . M . ( deraf lH M . Vand ) med 236,503 I . ( 1868 ) , inddeelt i 7 Fogderier , . , omfatter den sstlige Deel af Smaland faml Oland , og har foruden C . Staderne Vestervik , Vimmerby , Oscarshamn og Borgholm , samt 9 Flekker , Monsteras , Pataholm , Pastalla-

<h4>318</h4>Gab og sin frygtelige Bevabning af skarpe , trekantede, i Kanten fiint takkede Tander ; den tilhorer de varmere Have , isar Atlanterhavet . Hertil hore de store Haier , der fslge Skibene paa deres Fart for at sluge Lig eller Andet , fom maatte blive kastet overbord , og fom mange Steder gjsre det farligt at bade sig , da de kunne sluge et Menneske heelt og holdent eller ialfald afbide Arme eller Been ved et eneste Vid af deres skarpe Tandrakke . Lugten af Blod eller af Aadfler antages at hidlokke dem langveis fra . varoillll » , Slagt af Krabbernes Familie , hvortil vor almindelige gronne Strandkrabbe ( 6. MNN2B ) , der lever ved vore Kyster imellem Tang og Stene af Aadfler 0. dcsl . , hsrer . Skjoldet er fortil udtakket i 5 Takker paa hver Side ; Bagbenene ere Svsmmebeen , d . e . det sidste Led er fladt og bredt , stjsndt ikke saa bredt som hos de egentlige Susmmekrabber . Den afgiver et af Mange yndet Fsdemiddel . Cardamome , Kapslerne med de deri varende Fro af forskjellige til Kryderurternes Familie henhsrende Arter . Den lille C . eller Mala bar-C . , fom anvendes hyppigst hos os som Kryderi , faaes af Nlsttarik , O , l-6 , 2 moinuin fra Malabarkysten ; den lange C , af aiouratiouiQ fra Ostindien ; den runde C . af fra Sundaserne og Molukkerne, og den store C . af H . , fra Ostindien og Madagaskar . C . finder isar Anvendelfe fom Kryderi , fjelden fom Lagemiddel . Cardanus ( Hieronymus ) , beremt Mathematiker, Lage , Naturforster og Philosoph , f . 1501 i Pavia . Han bleu 1534 Professor i Mathematik i Mailand , hvor han ogsaa virkede som Larer i Medicin og som Lage . En Indbydelse fra Kongen af Danmark til at overtage et Professorat i Kjsbenhavn afslog han , efter Sigende af Hensyn til Klimaet og Landets Religion . Efter at have helbredet Hamilton , Erkebiskop af St . Andrews og Skotlands Primas , vendte han rigelig belsnnet tilbage til Mailand , blev senere medicinsk Professor i Pavia og derpaa i Bologna , hvor han larte indtil 1570 , da han paa Grund af en nretfaroig Anklage blev fangslet og led haard Behandling . 1571 blev han lss°ladt og reiste til Rom , hvor han sik eu Pension af Paven . I Rom endte han 1575 sit Liv ved en frivillig Hnngersdsd for ikke at overleve det af ham felv forudsagte Dsdsaar . C . var en vild Charakteer , som i fuldeste <b>Maal</b> reprafenterede Datidens Oplosning og Gjaring , Han har bestredet sit eget Liv i » ve vita proMa « , hvori han aflagger de utroligste Selvbekjendelser . Han fortaller , at hans <b>Moder</b> ssgte at fordrive ham allerede fom Foster , og at han fenere holdt sig ilive , uagtet hans Amme dsde af Pesten . Hans Liv var en Kjede af ydre og huuslige Ulykker , hans Sind vildt og damonist , hans Charakteer sammenfat af lntter Modsigelser . I » Le vita propria « siger han om sig selv , at han af Naturen har en philosophist og til Videnskaberne dannet <b>Aand</b> , er klsgtig , elegant , anstandig , vellystig, oprsmt , from , trofast , en Ven af Viisdom, tanksom , foretagende , larvillig , forstagen , listig , bitter , Religionsforagter , hevngjerrig , misundelig ofo . Hans ene Son havde forgivet sin <b>Kone</b> , fom var ham utro , og blev derfor henrettet; den anden Son lod han selv Orerne afhugge

<h4>2434</h4>den crldre Tids Forfsg i denne Retning maatte lade Meget tilbage at snste i denne Henseende til indre Sandhed og til ydre Trostab i Opfattelsen af Nordens Oldtid . Ogsaa Fremstillingen af den hjemlige Natnr og af Nordens Folkeliv var forssgt ; men Italien havde alligevel en stsrre Tiltrækningskraft . Under den startere aandelige Bevcegelse i national Retning , som betegnede Tiden henimod 1848 , vaktes ogsaa en mere levende <b>Bevidsthed</b> om Kunstens og Nationalitetens gjensidige Betydning for hinanden , i Særdeleshed af den larde og veltalende Kunstforster N . Hsyen . Han fremhævede den kunstneriske Forstaaelse af og Fslelfe for Nordens Nutidsliv og dens Natur fom en nsdvendig Forudsatning for Skildringen af Oldtiden . Den af Hsyen udtalte Tanke og den kunstneriske Retning , som fulgte med den , havde forsvrigt noget ganske Tilsvarende i Europas andre Lande ; thi allevegne gik Kunsten fra at vcrre romersk mere og mere over til at blive national ; og den havde desuden fin nsdvendige Forudsætning i den Eckersbergske naturalistiske Retning : udeu denne vilde den have strandet paa en Umulighed . Nu sluttede mange Kunstnere sig med Begeistring om den , fllllmeget mere som den aldrig optraadte erclusiv national eller forfsgte paa at hcrvde de nationale Fremstillinger som det Eueberettigede i Kunsten . Allerede tidligere havde Freund udfsrt fin magtige og phantasifulde Frise af Ragnaroksmythen; Bissen , hvis Virksomhed tidligere meest havde varet optaget af antike Emner , udfsrte sine Statuer af nordiske <b>Heltinder</b> til Christiansborg og efter Krigen et Antal betydelige nationale Monumenter , navnlig Landsoldaten til Fredericia; L. Frolich og Const . Hansen tegnede og malede Optrin af Nordens Gudeliv , og Marstrand, i hvis Kunst allerede de vittige Billeder af holbergste Scener havde vundet en Plads ved Siden af det italienske Folkeliv , malede endelig sine storladne , dramatisk levende Billeder af Christian IV.s Historie i hans Capel i Roskilde . Imidlertid dyrkedes dog de nationale Emner , som af Hsyen mere vare fremhævede fom Forudsætninger for Fremstillinger af Nordens Historie og Oldtid , med mere Forkærlighed end disse selv . I Figurmaleriet bleve Billeder af det danske Bondeliv de kjareste og hyppigst behandlede Opgaver ; Sonne og Roed leverede Billeder i denne Retning , og yngre Kunstnere , som Exner , Vermehren , Dalsgaard og C . Bloch , samlede fortrinsviis deres Virksomhed om disse Emner ; Bloch gik dog senere over til en Behandling af mythiske og historiske Emner i stsrre Stiil . Sonne og Simonfen skildrede i flere Billeder de slesvigske Kriges Begivenheder . Landskabsmaleriet var allerede efter Eckersbergs Tid udviklet videre , isar af Ksbke ; men henimod 1848 fremstode Kunstnere, som ved indtrcmgende Fordybelse i D.s Natur grundlagde en ny Skole i Landskabsmaleriet, navnlig den hsitbegavede Dyr- og Landskabsmaler I . T . Lundbye , P . C . Skovgaard og G . Rump ; siden den Tid har en Mcengde yngre Krcrfter helllget sig til denne Green af Kunsten . Ogsaa Ssmaleriet , Dyrmaleriet , Blomstermakeriet og Architekturmaleriet have fundet talentfulde Dyrkere . I Billedhuggerkunsten fik Bissens Virksomhed isar Betydning for den yngre Slcrgts <b>Uddannelse</b> ; ved hans Side stod lerichau

<h4>2547</h4>Velfardsudvalget , men i Sept . 1793 afslog han Gjenvalg . Da Girondinerne stadig rettede Angreb paa ham som Ophavsmanden til Septembermyrderierne, fremkaldte D . Oproret 31 Mai 1793 , der medforte deres Udstodelse af Conventet , sijondt han ellers ingenlunde oustede at rydde dem af Veien , men netop fredelig Samvirken . Skjsndt det halve Europa og det halve Frankrig reiste sig mod Revolutionen , holdt D . Modet oppe og har narmest AZren for det energiske . Forsvar . Han spottede med dem , der hyldede Fornuftens Gudinde , og hjalp med at bringe Hebert og hans yderliggaaende Meningsfllller paa Skafottet ; men da han nu troede <b>Maalet</b> naaet og vilde standse Strommen , vakte det Rodespierres Uvillie og Skinsyge , og da man advarede D . mod hans Hevn , afviste han det , da han „ hellere vilde blive guillotineret end vcrre Guillotineur " . 31 Marts 1794 lod Robespierre D . og hans Venner fangste og anklage som „ Mlllldeholdne " , og da han frygtede D.s Veltalenhed, lod han Conventet forbyde dem at forsvare sig nnder Paastud af , at det vilde fremkalde Forstyrrelser fur Domstolen . D . erklarede stolt , at hans „ Navn vilde leve i Historiens Pantheon " , og fpaaede , at „ de feige Rovere , for hvem man nu offrede dem , ikke lcrnge skulde nyde Seireu " , samt at han „ vilde trakke Robespierre med sig i Faldet " . 5 Apr . 1794 blev D . henrettet, og paa Skafottet fremkaldte Tanken om hans unge <b>Hustru</b> en oieblikkelig Klage , som han dog tilbagetrcrngte med Ordene „ Ingen Svaghed " . D . var en Mand af kcrmpemcrssig Bygning med grove og stygge Trak ; hans Sind var lidenskabeligt, og hans voldsomme Fagter og tordnende Stemme indjog Skral og fremkaldte Lydighed ; men i sit private Liv var han godmodig og mild . Han lod sig let overtale til at <b>vise</b> Skaanfel mod den Enkelte , var hengiven til Forlystelser , og tog bverken i Betantning at modtage Penge fra Hoffet eller senere at berige sig ved Eonfistationer. Man har med Rette kaldt ham „Mangdens Mirabeau " og sammenlignet dem ; de havde begge samme Laster , samme Energi , samme Veltalenhed, men hos Mirabeau stak Patricieren , hos D . Pledeieren igjennem ; hiin mindede om Catilina , denne om Marius . Danzig , starkt befastet Stad i den preussiske Prov . Preussen ved Weichselflodens vestl . Arm , 1 M . fra Osterfoen , 55 M . n . s . for Berlin . 89,311 I . ( 1867 ) . D . er en gammeldags bygget Stad med talrige smukke Privatbufe ; blandt dens offentlige Bygninger fremhaves Mariakirken , opfort 1343 - 1503 ; Naadhuset fra det 14 de Aarh . ; Artushof fra det 16 de Aarh . , benyttes nu fom Bors ; Toihuset og flere af Fastningsportene . Industrien staner paa et hoit Trin ; dens vigtigste Frembringelser ere Sukker , Skibsbrsd , Stivelse, Tobak , Spirituosa , chemiste Praparater , Potaske , Cichorie , Guld- , Selv- og Navvarer , Vl og Meel . Handel og Skibsfart er meget betydelig ; de vigtigste Udforfelsartikler ere Tommer, Korn og Oliesro . D . , der allerede navnes i det 6 te Aarh . , var i det 10 de Aarh . en betydelig Stad , hvis polske Navn var Gdansk . 1310 kom den under de tydste Riddere og var siden en af Hllnseforbuudets 4 Qvarteerstader . 1454 unddrog den sig Riddernes Herredomme og kom ved Freden i Thorn 1466 under Polen , dog

<h4>2828</h4>ved Freden i Aachen , hvorved Ludvig beholdt 12 befastede Stader i Nederlandene , men tilbaqeqav det erobrede Franche-Comte . Devon , Shire i det fydvestlige England mellem Bristolkanlllen og den engelske Kanal og forsvrigt omgivet af Cornwall-Shire mod B . og Somerset- og Dorset-Shire mod V . 121 « M . med 584,373 I . ( 1861 ) . Kysterne paa begge Sider ere nasten overalt hsie og klippefulde, og danne talrige Bugter og Indstjaringer , blandt hvilke maae fremhaves den store Varnstaplebugt paa Nordsiden og Torbai paa Sydsiden, samt Havnene ved Ermouth , Teignmouth , Dartmouth , Portsmouth og Plymouth . Shirets Indre er et Hsiland , der mod V . i en mosedakket Granitdannelse haver sig indtil 2000 F . De vigtigste Vandlob ere Are , Otter , Ex , Teign , Dart , Avon og Tamer , der alle falde i den engelske Kanal , og Taw og Torridge , der forenede falde i Barnstaplebugten . Klimaet er paa Sydkysten meget mildt , men fugtigt ; i Hsilandet er det temmelig strengt , men snndt . I Landbruget spiller Kvagavlen Hovedrollen , baade som Meieridrift og Tillag af Slagtekvag ; Faareholdet er meget stort ; Kornavlen er i Fremgang , men Shiret staaer dog i denne Henseende temmelig langt tilbage . Mineralproduktionen er af meget stor Betydning , og omfatter navnlig Kobber og Tin , famt noget Jern og Bly ; store Masser af stint Pottemagerleer udfsres uårlig til Stafford- Shire og andre Egne , hvor Fajencefabrikationen har vundet Fremgang . Den vigtigste Industrigreen er Fabrikation af Uldstoffer . Hovedstad Exeter . Devon , Nord- , en stor bjergfuld ( indtil 2000 F.s Hside ) i Nordpolarhavet , stilles ved Lllncastersund og Varrowstradet fra Baffinsland , ved Wellingtonkanalen fra Ven Cornwall og ved lonessund fra Nord-Lincoln . Paa Sydvestkysten ved Varrowstradet er Erebusbugten , hvor Franklin overvintrede 1844 — 45 og Pulleu 1852 og 1854. Devon ell . Devonshire , siden Kong Henrik I.s Tid Navnet paa flere engelske Adelsflagter , af hvilke de meest bekjendte ere Familien Courtenay , som 1553 af Dronn . Marie , og Cavendifh , fom 1618 af Kong Jakob I . sil Grevetitel efter Landskabet D . , hvorefter stdstnavnte Familie senere fik Hertugtitel , idet William Greve af D . blev af Kong Vilhelm Hl . af Oranien , for hvis Tronbestigelse han havde varet meget virksom , ophsiet til Hertug af D . ; han dsde 1707 som Dronn . Annas Ollerhofmester . William , den femte Hertng af D . , f . 1748 , blev 1766 udnavnt til Storstatmester af Irland , hvor et Par af de foregaaende Hertuger blandt hans Forfadre havde varet Vicekonger , og han vedblev som oprigtig Whig ligesom disse at misbillige Hoffets Politik imod det ulykkelige Land ; d . 1811. Han var fsrft gift med Georgina Spencer , s . 1757 , en ved Skjsnhed og Elstvardighed ikke mindre end ved <b>Aand</b> og <b>Dannelse</b> udmarket <b>Kvinde</b> , der ogsaa besad Poetisk Talent og tog levende Deel i politiske Anliggender ( da hun engang tilligemed andre <b>Damer</b> arbeideoe for Fox ' s Valg til Parlamentsmedlem for Westminster, forlangte en <b>vis</b> Fleischer et Kys af hende for sin Stemme , fom han ogsaa til almindelig Morflab fik af sin Tids stjsnneste <b>Kone</b> ) ;

<h4>3128</h4>2806 li ! M . med 949,682 I . ( 1864 ) . Landet er en flad , af Don og dens Bifloder gjennemstrsmmet Slette . I dets vestlige Deel findes ikke faa tydske Kolonister , der med Held dyrke Hvede og Biin , ligesom ogfao . Kosakkerne , der tidligere knn dreve Hefte- og Kvagavl , pac , mange Steder have efterfulgt Kolonisternes Erempel . Foruden den egentlige Kosakbefolkning , der er deelt i 7 Commandoer, leve paa Stepperne c . 26,000 Kalmuker samt mindre Dele af forskjellige andre Stammer som Nomader , Hovedstad Novo- Tfcherkafk . Doolin af Mainz , en af Heltene i den carolingiste Sagnkreds , jfr. Alxlnger . VoomBs2 . ? book , f . Domes < l2vl > ooK , Doornik , s . Tournay . Doppia , en Guldmynt i Sardinien 20 Lire ell . 20 Frc . ' s , c . 7 Rd . 9 Sk . dansk . lloppio , dobbelt , s . Er . il 6. movimento ( ell . I ' iztesso tempo ) , som ved Taktforandring i samme Stykke i de fleste Tilfalde angiver , at Tattdeelnoden ( i Takt Halvnoden , i Takt Fjerdedeelnoden, i tz Takt Treottendedeelnoden ) i den sidste Taktart skal udfores i famme Tempo fom Taktdeelnoden i den fsrste . Doppler ( C . ) , tydst Mathematiker og Physiker , f . 1803 , d . 1853 , blev efter at have bekladt forskjellige Lærerposter 1851 Professor i Erperimentalphysik ved Universitetet i Wien . Hans Skrifter , som angane Mathematik , Physik og Astronomi , ere for en stor Deel spredte omkring i forskjellige Tidsskrifter . Dora Baltea , D . Riparia , f . Po . Dorade , 1 ) d , f.f . Guldbrafen , en i Middelhavet levende , fslvfarvct Fist med et halvmaaneformigt , guldguult Baand mellem Vinene ( 6 nrvBopllrvB aurata ) , af Sparoidernes ( Haubrasenerues ) Familie. 2 ) Doi-2 , 2 , sydamerikansk Mallestagt med paudsret Hoved og en Rakke af holede eller piggede Skjolde langs med Kroppens Sider . Ogsaa om Guldfisken gurlitus ) og om visse store Makrclfisie ( Dolfiner ) bruges Benavnclsen D . Dora d ' lstria , Forfatternavn for Fyrstinde Helene Kolzov-Massalfiy , s . 1828 i Bukarest . Hendes Fader Fyrst Mich . Ghita var en Tid lang Indenrigsminister i Balachiet ; to af hendes Onkler have vcrret Hofpodarer . Den larde Grater Pllppadopulos gav hende en udmarket og alsidig <b>Dannelse</b> ; 1841 — 48 opholdt hun sig i Udlandet, vafentlig for sin Aandsudviklings Skyld . 1849 giftede hun sig med den russiske Fyrste Kolzou-Massalsty , fulgte ham til Rusland , og tog 1855 efter en Reise i Schweiz og Italien Ophold i Livorno . Med stor <b>Dannelse</b> forener hun en frisindet og alsidig Livsbetragtning baade ligeoverfor kirkelige og sociale Spsrgsmaal . vie monaBtia.ue 6 » NB orientale « ( 1855 ) er en Polemik mod Munkelivet som Civilisationens Fjende i de romerske og grast-katholste Lande . I disse Aar strev hun ogsaa Artikler i liberal <b>Aand</b> i Turinerbladet » li « liritto « , der <b>vise</b> skarpt Blik for de romanske Stammers Eiendommelighed. » I.eB lemmes en Orient « ( 1860 ) og » VeB lemmLB par une temme « ( 1864 ) kampe for Frigjorelse af <b>Kvinden</b> i de ostlige Lande . I dorcl « les iaes nelvetiques » ( 1864 ) samlede hun en Rakke i « Nevue 6 es 6 eux moncie » « indrykkede Noveller . Pressen i mangfoldige euro-

<h4>3771</h4>Elisa , Prophet i Israels Rige under Kongerne loram og lehu ( 896 — 56 ) , Discipel af Elias , virkede i dennes <b>Aand</b> , ndrustet med lignende Undergaver , fremkaldte Straffen over Achabs Huns , og dsde 840 i Samaria under Kong Jons . Elisabeth , et fra Hebr . stammende Navn , der bet. den , der fvcerger ved Gud , Gudsdyrkerste , forekommer i det Gamle Test . ( 2 Moseb . Vi . 23 ) kun som Navn pall Aarons <b>Hustru</b> ( „ Eliseba " ) ; i det Nye Test . paa en Efterkommer af disse , Prcesten Zacharias ' s <b>Hustru</b> og Johannes Dsberens <b>Moder</b> , beslcegtet med Jomfru Maria og bekjendt af dennes Bessg efter Eugleaabenbariugen ( Luk . Iste Cup . ) . Elisabeth , den hellige , Abbedisse i Klostret Schonau , d . 1165 , var Prophetiude og revsede Geistlighedens Yppighed . Hendes Broder Ekbert fra Munkeklostret af f . N . , oversatte og offentliggjorde hendes Spaadomme ; ved hendes Visioner fik Legenden om Ursula og de 11,0 L0 Jomfruer sin <b>Uddannelse</b> og Stadfcestelse . Elisabeth , den hellige , Landgrevinde af Thiiringen, Datter af Kong Andreas 11. af Ungarn , f . 1207 i Presburg , blev allerede som lille Barn trolovet med den dengang lOaarige Landgrev Ludvig af Thuringen og opdraget fammen med ham paa Wartburg ved Eisenach . I sit 15 de Aar blev hun gift . Hele sit Liv offrede hnn til Kjccrlighedsgjeiuiuger og strenge Bodssvelser . Hver Skjcertorsdag vaskede hun 12 Fattiges Fsdder og gav dem Sslvpenge ; paa sit Legeme bar hun en Bodsdragt af Haar , og hun vidskede sig selv hver Fredag . For de Fattige var hun en sand <b>Moder</b> ; huu bessgte de Syge og pleiede dem og bragte de Nsdlidende Klceder og Fode . Da hendes LEgtefcelle syntes , at hendes Godgjsrenhed gik for vidt , og engang modte hende , fom hnn gik ud med Brod og Kloeder til de Fattige , fortceller Legenden , at disfe Ting forvandledes til Roser , da Greven vilde see , hvad hun skjulte i sit Forklcede . Med haus Tilladelse byggede hun paa Skraaningen af Wartburg et Hospital for Spedalske og stiftede tillige et Redningshuus for fattige forssmte Bsru . 1227 blev hun Enke og tilligemed sine 4 Born fordrevet fra Wartburg af siu Svoger , den raa Henrik Raspe . E . tog sin Tilflugt til Bistpppen af Bamberg , men bofatte sig senere i Marburg , hvor hun byggede et Hospital , i hvilket hun pleiede de Syge indtil sin Dsd 19 Nov . 1231. Pave Gregor ! X . erklcrrede hende 1235 for Helgeninde paa Grund af de Undere , der skete ved hendes Grav ; hendes Dodsdag er hendes Mindedag. Igjennem sin Datter Sophie blev hun Stammoder til det hessifle Fyrstehuus . Elisabeth , danst Prindsesse og Kurfyrstinde af Brandenburg , Datter af Kong Hans og Dronn . Christine og den eneste af dette Kongepars 5 Bsrn foruden Kong Christiern 11. , der naaede den vorne Alder , f . 4 Aar efter denne , 1485 , og ligesom

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010021603013'target=_>Arndt, Johann, 1855, Johan Arndt's Postille, eller Aandrige Forklaringer over de aarlige Søn- og Helligdages evangeliske Texter</a><h4>2495</h4>Hvor heinodvendig Guds Fred er os , det erfare vi , naar vi geraade i Bedrøvelse og Anfcrgtning , hvad enten det er legemlig eller <b>aandclig</b> , ind » vortes eller udvortes , og dersom Guds Fred da ikke opholdt os , saa maatte vi forsage . Desaarsag have vi stedse at igjentage St . Pauli Onste og Ven .  Og den Guds Fred , som overgaaer al Forstand , stal forvare eders Hjerter og eders Tanker i Christo Jesu , " Phil . 4 , 7. Naar vi belcrnke vort elcn « dige , urolige Liv og denne vor Udloendigheds Tid , saa rna « vi vel med Patriarken Jacob bekjende ,  at samme er kort og ond " ( 1 M 05 .47 , 9 ) , og med den hellige Guds Mand Moses sige , at  vort Liv ei er andet end Meie og Arbeide " ( Ps . 90 , 10. ) , med den hellige Mand lob maa vi sige ,  at vi ere i Strid paa Jorden , og at vore Dage ere som ln Daglonnas Dage " ( lobs B . 7 , 1. ) , og med den hellige Mand Syrach ,  at stor Urolighed er stabt for hvert Menneske , og at der er et tungt Aag pag Adams Born , fra den Dag de udkomme af deres Moders Liv , indtil den Dag de komme til Alles <b>Moder</b> igjen " ( Syr . 40 , 12. ) ; ja med den Herre Christus selv maa vi sige , at vi i Verden ei stal have andet end Angest , Joh . 16 , 33. Besynderlig vil i de sidste Dage Sorg , Frygt , Angest , Banghed , Bcrven , samt udvortes og indvortes Uro og Ufred tåge mere og mere Overhaand , og vil ei alene den udvortes legemlige Angest , som kommer af Armod , Ned , Mangel , Hunger og Torst , Frost og Nogenheo , Dyrtid , Krig , Pestilentse , Forfølgelse og Fcrngsel , blive almindelig for Verdens Ende ; men denne udvortes Nod bebuder os en indvortes Sjcrle-Nod og Mangel , hvoraf Mange vil komme til at lide : — Her vil findes En , som ti vil kunne troe , hvem Vantroe og Fortvivlelse vil plage ; hist vil vare En , som ei vil kunne bede ; — ber vil En blive anfcrgtet af Djcrvelen , saa at Haabet udslukles i ham ; hist vil En verre , hvem Djawelrn vil plage med gudsbespottelige Tanker og med sine gloende Piles Indskydelse i Hjertet ; - - atter en Anden vil kjampe med Fortvivlelse , og kort , det crgyptiste M < m7.l > i hvilket mange Djirvrls-Larver vil <b>vise</b> sig , og forstroekke og qvcrle Sjcrlene , vil overfalde Verden for Guds Berns endelige Forlssning , ligesom det stedte i A5gYP < ten . Falsk <b>Lcrrdom</b> er det aandelige , crgyptiske Morke , og Kjcrtterie og Anfcrgtninger ere Djaevels-Larvene . Og visseligen gaacr dette for sig nuz

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008092303015'target=_>, 1854, Eenfoldige Betragtninger over alle Søn- og Fest-Dages Evangelier, for største Delen udarbeidede efter en i Sverige under ovenstaaende Titel udgiven Postille</a><h4>924</h4>2 ) Dog er det Me saa farligt , som det kunde synes , thi Herren er trofast , og stal ikke lade os fristes over vor Formue , i ) Han selv skjenker <b>Sjelen</b> et saadant <b>Maal</b> af Styrke , som er fornsdent til at kunne bestaae den Prove , han setter den paa , saa at dens provede Troe virkelig kan seire . Den Cananeiste <b>Qvinde</b> tabte ikke Modet , og lod sig Me engang afvist ved det haarde Svar , da Frelseren sagde : " Det er ikke smukt , at man tager Brodet fra Bornene og kaster det for smaae Hunde . " Hun vidste strax at benytte sig just af det tilsyneladende Haarde og Afvisende , som laae i dette Svar . Hun tog Frelseren paa Ordet , hun vilde gjerne finde sig i hans Sammenligning

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006081400005'target=_>Nissen, Hartvig, 1876, Afhandlinger vedrørende det høiere og lavere skolevæsen</a><h4>89</h4>Nissen indgik den 12 te Juni 1843 i Ægteskab med Karen Magdalena Aas , Datter af afdøde Sorenskriver Johannes Henrik Aas og <b>Hustru</b> Stine Colban Dette Ægteskab velsignedes med 12 Børn , livoral 9 leve . Ligesom Nissen i sit lykkelige Familieliv spredte Mildhedens og Kjærlighedens <b>Aand</b> udover sin huslige Kreds , saaledes vandt han ogsaa i sin hele Færd som Menneske alle sine Omgivelsers Hengivenhed og Erkjendtlighed . Hvad enten han som Bestyrer al sin Skole optraadte med Ros eller Formaning og Straf , eller han om sin gjestfrie Arne samlede en Kreds af hengivne ældre og yngre Venner ( og særlig skal vi minde om det vakre Træk , at han som Skolebestyrer aabnede sit Hus lor de mange Gutter , der var komne til hans Skole fra andre Kanter af Landet og i Hovedstaden intet Hjem havde at ty til ) , hvad enten kan stredes med Aandens Vaaben for sit Livs <b>Maal</b> eller han hvilede sig fra Kampen i sit lykkelige Hjem , alticl og « Ilevo ^ ne var han dog den samme milde , humane og vennesæle Mand , for hvem Sag var mere end Person , og Hjerte mere end Stand og Stilling . Og derfor udgaar der fra Hartvig Nissens Minde en kraftig Opfordring til den yngre Slægt om at virke i Arbeidslyst og Styrke , i Tro og Fa ; 6 rellln6BBinH som han , Aandens 8 t ! - i68man « i med det uplette6e Skjold . ,

<h4>2926</h4>andre Fag , , der indeslutter en ligesaa stor , om ikke større dannende Kraft ! ' ) Det vil derfor være saa meget mere vendigt noget nøiere at undersøge Sprogdannelsens Forhold som Element i den <b>kvindelige</b> <b>Dannelse</b> , og derefter bestemme Sprogundervisningens Betydning og Udstrækning i Pigeskolen . I den hele Tilværelse indtager Sproget en særegen , men betydningsfuld Plads . Det er ikke ligesom Naturen udgaaet umiddelbart fra Skaberens Haand ; det er heller ikke et Produkt af Kunsten . Det er en Affødning af den menneskelige <b>Aand</b> , der først gjennem denne Bin Skabning bliver istand til at udfolde sig . Uden Sproget vilde de menneskelige Aander ikke danne en Menneskeaand , og , da det kun er ved Delagtighed i den fælles Aands Liv , at den individuelle kan udvikle sig og naa høiere Fuldkommenhed , saa betinger Sproget ogsaa Muligheden af en Udvikling for Individet . Men er nu Sproget for Mennesket en saa vigtig vel af Tilværelsen , saa har det ogsaa Krav paa dets Interesse og Opmærksomhed , og , naar vi ville hæve os til en klarere IndßiFt i den hele Tilværelse , til en høiere <b>Dannelse</b> , kunne vi ikke undlade ogsaa at søge at kaste et Blik ind i dette Gebet af Tilværelsen , for at lære Sprogets VBLB6N at kjende , og gjennem denne dybere Indsigt forskaffe os en større Færdighed og høiere Fuldkommenhed i at benytte det til Udtryk for vor egen individuelle Aands Ytringer . Allerede denne Betragtning indeholder formentlig tilstrækkelig Begrundelse ak den Anskuelse , at Sprogundervisningen bør spille en ikke uvigtig Rolle i den almindelige Dannelses Skole ; men den formaar derimod ikke at besvare Spørgsmaalet om , hvor vidt denne Undervisning bør gaa , over hvor " mange af de enkelte Bprog , hvori Bz ) rogid66u aabenbarer sig , den bør udstrækkes. En videnskabelig Undersøgelse af det Sprogstof , som vi allerede besidde ved en umiddelbar Tilegnelse , Modersmaalet nemlig , vil allerede være tilstrækkelig til at skjænke 08 Indsigt i Alt , hvad der er væsentligt ved Sproget og forskaffe 08 hin praktiske Færdighed og større Fuldkommenhed i at benytte det , hvorvel Indsigten maa vinde i Styrke og Klarhed , og Færdigheden derigjennem forøges , naar Modersmaalets Phænomener

<h4>2938</h4>Den Del af Tilværelsen , hvis nøiere Betragtning og Undersøgelse jeg hidindtil har fremstillet som Skoleundervisningens Gjenstand , er Menneskelivet og dets Forhold til Guddommen ; men ogsaa den anden Hoveddel af Tilværelsen , Naturen , har et uafviseligt Krav paa Skolens Opmærksomhed . Beskaffenheden af den Jord , der er Tumlepladsen for Menneskets travle Færd , betinger i høi Grad Menneskelivets Retning og Charakter ; de levende Væsener , som omgive os , Træ- og Plantevegetationen og Naturforholdene overhovedet indvirke jo mægtigt paa Livets Gestaltning . Det maatte derfor være en Trang hos os at lære disse Gjenstande at kjende , selv om denne <b>Kundskab</b> ikke var af Vigtighed for desto lettere at kunne beseire de Vanskeligheder, som Naturen lægger i Veien for Menneskelivets Udvikling, og for desto omhyggeligere at kunne udfinde og benytte de Hjælpemidler , som den dertil yder . Naturens Gjenstande ere imidlertid saa mangfoldige , at det langt overstiger den individuelle Uenn6Bkeaandß Kraft at lære dem enkeltvis at kjende og fatte deres Sammenhæng . Selv den , der kun , i Kraft afden almindelige Vannelßeß Krav , vil med sin <b>Aand</b> gjennemtrænge det Væsentligste , det Betydningsfuldeste i denne Del af Tilværelsen, maa derfor indskrænke Big til enkelte Gruppers nøiere Udforskning . Denne af Nødvendigheden paabudne Indskrænkningenß 6 rundßNtning maa især Skolen strængt iagttage , da den langt sikrere opnaar Bit <b>Formaal</b> ved at meddele Lidet , men grundigt , end Meget , men overfladisk , og navnlig maa Pigeskolen være forsigtig i denne Henseende , og vel prøve baade den <b>kvindelige</b> Dannelses Fordringer og sin egen , af de ydre Forholdes Magt betingede , Evne til at tilfredsstille dem . En saadan

<h4>2947</h4>Jeg har nu i det Foregaaende søgt at <b>vise</b> , at hvilke Elementer den høiere <b>kvindelige</b> <b>Dannelse</b> bestaar . En saadan Udvikling har jeg anseet for aldeles nødvendig ikke blot for med nogen Sikkerhed at kunne bestemme , kvilke Fag der bør optages som Undervisningsgjenstande i Pigeskolen , men ogsaa for at de optagne Fags større eller mindre Betydning og deraf betingede Udstrækning skal kunne stilles i et tilstrækkeligt klart Lys . Jeg har imidlertid tilfulde følt Vanskeligheden al at forbinde den videnskabelige Stringents og den Fuldstændighed , som Opgavens Natur gjør Krav paa , med de Fordringer til Form og Udstrækning , som ydre Forholde for Tiden opstille for mig . Har jeg desuagtet været istand til nogenlunde fyldestgjørende at løse min Opssave , maa det iallalH fremtræde som et sikkert Hovedresultat , hvorpaa der kan bygges videre , at den høiere Pigeskoles Undervisningsgjenstande ere : . Religion , Historie , Modersmaalet med Svensk og dets Literatur , Tydsk , ( Fransk , Engelsk), Geographi , Naturhistorie , o : især Botanik , Regning , Bkrivning, Sang og Tegning samt desuden Haandarbeider .

<h4>2949</h4>I det Foregaaende har jeg søgt at <b>vise</b> , hvorom den høiere Pigeskoles Virkßomlied som Undervisningsanstalt især bør dreie sig . Resultatet af en saadan Undervisning Bom den , jeg har antydet , vil forhaabentlig være en harmonisk Udvikling af det hele intellektuelle Menneske , og Skolens nærmeste <b>Formaal</b> som Undervisningsanstalt vil saaledes derigjennem kunne naaes . Men Skolen skal ikke blot være en Undervisningsanstalt ; den skal være en Dannelsesanstalt , og som saadan maa den omfatte <b>Dannelsens</b> hele Anliggende og altsaa gjøre Gemyttets og Charakterens Udvikling eller , hvad man i indskrænket Forstand kalder , Opdragelsen til sit <b>Formaal</b> . Dette er ogsaa allerede udtalt i den tidligere Del af min Afhandling , hvor jeg viser , at det er Skolens Opgave at fremkalde og styrke - Religiøsiteten ; thi hvor sand og ægte Religiøsitet findes , der er den Grundvold lagt , livorpaa en riZtiZ og sund Udvikling at Gemyt og Charakter ene kan bygges . Men med Grundvolden er dog Bygningen ikke færdig . Skolens Bestræbelse maa derfor uafbrudt gaa ud paa ' at fremkalde hos sine Lærlinge alle <b>kvindelige</b> Dyder . Det er imidlertid ikke min Hensigt her at give en fuldstændig Fremstilling at , hvordan Skolens Virksomhed i denne Retning kan og bør være ; jeg anser det for tilstrækkeligt kun at give nogle Vink . — At den hele Undervisning er et vigtigt , et særdeles vigtigt Middel til at fremme . Opdragelsens <b>Maal</b> er indlysende ved sig selv . Dog er det ikke blot Undervisningsstoffet , ikke blot de Grundsætninger og Anskuelser , som meddeles ved og gjennem det , ikke blot de opmuntrende og advarende Exempler , som det yder , der virke til Fremkaldelsen af de <b>kvindelige</b> Dyder og til deres Forædling ; men ogsaa selve Undervisningsmaaden , Methoden , har i denne Henseende stor Betydning . Er der Fasthed, Alvor og Orden i den , da tjene disse dens Egenskaber ikke blot til at fremme Grundighed i <b>Kundskab</b> og Indsigt ; men de slåa ogsaa Rod i Lærlingens Sjæl . Men især kan Undervisningen ved at meddele Indsigtens Elarhed og Grundighed modvirke Übeskedenheds og Indbildskheds Indtrængen i Gemyttet. Og kan det ikke negtes , at vor Tids hele aandige Tilstand er en frugtbar Jordbund for disse Egenskaber til at voxe i , saa bør Opdrageren være desto omhyggeligere med at opluge dette Ukrud . Og dog hører man ofte Klager over , at den <b>kvindelige</b> Undervisning netop fremkalder og nærer disse Egenskaber . Forsaavidt imidlertid saadanne Klager ikke bero paa en Misforstaaelse , idet man forvexler Paavirkninger andetstedsfra

<h4>2951</h4>med Undervisningens , da kunne de blot have sin Grund deri , at Undervisningen er let , flygtig og overfladisk istedetfor alvorlig , mdtrængende og grundig . Det er jo paa Vandets Overflade, at Boblerne spille . En saadan Undervisning , som den jeg i det Foregaaende nar fordret , vil derimod visselig nedlægge den sande Beskedenheds Sæd ; thi imedens hint Skin af Dannelse, Hen overfladiske <b>Kundskab</b> , fremkalder Indbildskhed , Hovmod og LeßtrNdelß6n ekter at synes u6en at være , er det netop den sande og grundige Viden eller Indsigt , der er Kilden til den sokratiske <b>Bevidsthed</b> , om Intet at vide og til Bestræbelsen efter at være , men ei at synes . I nær Forbindelse med Beskedenhed staar den ydre Høflighed . Den er derfor heller ikke just vor Tids Barn , skjønt den er en sand Pryd for <b>Kvinden</b> , dog mere som Tegn paa den indre Beskedenhed end for dens Værd iog for sig . Forsaavidt som den ikke direkte uHg ^ Her af Beskedenheden , fremmes den vel imidlertid mindre ved Undervisningen end ved Skolens Paavirkning iøvrigt , hvilket ogsaa i større og mindre Grad gjælder om de øvrige kvinHeliZe Dyger. Derfor er ogsaa Disciplinen og alle de Forholde , som dermed staa i Forbindelse , af høi Vigtighed for Opdragelsens Element i Skolens Virksomhed og fortjener ogßaa her for et Øieblik vor Opmærksomhed .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016051048082'target=_>, 1878, Huus-Postil, eller En Samling af forskjellige Forfatteres, meest Luthers, Franckes og Pontoppidans Betragtninger over Søn- og Festdages deels Høimesse-, deels Aftensangs-Texter : samlede til en Aargang, og udgivne i den Hensigt, at befordre og vedligeholde christelig Huus-Andagt</a><h4>2876</h4>2. Denne Forfarenhed raadcr jeg nu ei til at oppebie , eller lade det komme an paa , efterdi Sagen er ei af de smaae og ligegyldige Sager , men saare vigtig og betydelig . Her <b>moder</b> os vor Betragtnings andet Hoved-Punct . Denne grunder sig paa de alvorlige Formanings-Ord , som Apostelen hcrftcr til sin <b>Lærdom</b> , for at indstjærpe den i vor Samvittighed , nemlig , for han siger : Hvad et Menneske saaer , det stal han hoste , saa hedder det : Farer ikke vild , Gud lader sig ikke spotte . Er det ei saamcget sagt : denne er en Guds bespottelig Vildfarelse , hvis I negte eller modsige den Sandhed . Vel sindes der andre Sandheder ligesaa visse som denne ; dog kunde nogle af dem Være ei nær saa vigtige , betydelige og magtpaaliggendc , saaat , om man jo tvivlede , eller tog feil , da havde Vildfarelsen ei strax saa farlige Følger . Ja visselig , stjondt al Sandhed beholder sin cedle Priis , saa dog er det en forholdsmæssig Priis , alt efter Tingenes Vigtighed , og man kan negte eller forfcgtc visse blot speculative Lærdomme langt ivrigere , end det , man strider om , er vcerd . Dette var meget langtfra Pauli Sind : han trcettede ei om blotte Ord , efter nnyttige og übetydelige Spo ' rgsmaal . - Men naar Talen var om Ting af Vigtighed , saasom her , da brugte han al Iver og Eftertryk , for at scette Menneskets Sind i Vevcegelse og Ngtsomhed . Naar han maatte frygte for , at sikkre Mennesker kunde gjore sig selv og andre <b>vise</b> , der var iugen Host at vente efter Sceden , ingen Evigheds LM efter Tidens Arbeide , eller dog ingen saadan Lon , som svarede til Arbeidet , da saac han vel , at denne Vildfarelse rev Grunden om , og var ligesaa vigtig , som hvis man negtede Gud eller i det mindste hans Egenskaber , om ikke hans Vasen ; thi ingen Gud , eller en Gud , som ikke straffer og belonner , synes ham ligemeget . Denne Vigtighed ligger i Ordene ; Farer ikke

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009073000103'target=_>Wexels, W.A., 1843, Forklaring over Det nye Testamentes Skrifter</a><h4>1182</h4>Virkningen givet , om ben end ei strax fremtrceder . . Saasnart et Menneske i Hjertet troer og tilegner sig Christus som sin Rctfcrrdighed , saa er det retfcrrdigt , og i Grunden altsaa tillige saligt ; men Saligheden betragtet som noget i en stadig <b>Bevidsthed</b> Fremtraadt , som noget Fuldendt , som en uforstyrret Freds- og Glaedss-Tilstand , er endnu ikke tilstede , noget Fremtidigt , og krcrver , for at opnaaes , Troens Udbrud i levende Bekjendelse ; jo mere Troen i denne gaacr frem til Seier over Verden , Djoevclen , Synden og Dsden , des saligere fsler Mennestet sig , og , naar Bekjendelsen ikke <b>moder</b> flere Hindringer , men Hjertets Liv udstrsmmer i den aldeles übundet som jublende Lov og Priis for Guds Throne , da er Saligheden fuldendt. En Tro derimot ? , , der blev uden Bekjendelse , hvad der egentlig er det samme som uden Selvbevidsthed , vilde efterhaanden udstukkes igjen og aldrig naae Saligheden , Troens <b>Maal</b> . Sml . ellers Matth.l o , 32. At bekjende den Herre lesum er naturligviis at bekjende lesum som Herre ( Phill . 2 , 11. ) d . e . soni Forleser , som den , der ved sit Blod har kjsbt sig et Folk til Eiendom , som den , der som Menneskcns Son er hsit ophsiet over alle Ting , har Almagt i Himlen og paa Jorden ( Ap . Gjem . 2 , 36. ) .

<h4>1609</h4>Elskelige i HErren . 9. Hilser Urbanus , vor Medarbeider i Ehristo , oa Stachys, mln Elskeltge . Itt . Hilser Apelles , den Prsvede i Christo . Hilser dem , som ere af Arlstobult ( Huus ) . 11. Hilser Herodion , min Frcende . Hilsw dem af Narnssi lHuus ) , som ere i HErren . 12. Hilser Tryphena oa Trnphosa, som have arbeidet i HErren . Hilser Perfis den Elskelige , som bayer arbeldet Meget i HErren . , 3. Hilser Rnfus , den Udvalgte t HErren . oa hans og min <b>Moder</b> . 14. Hilser Asyncritus , Phlegon , Hermas , Patrobas Hermes , og Brsdrene hos dem . 15. Hilser Philologus Vg lulias , Nereus og hans Sester , og Olympa . og alle de Hellige hos dem . 1 « . Hilser hverandre med et helligt Kys . Chnsti Menigheder hllse eder . 17. Men jea formaner eder , Brsdre ! at I give Agt paa dem , som volde Splid og Forargelser tvertlmod den <b>Lcrrdom</b> , som I have lcrrt ; og viger bort fra dem . 18. Tht Saadanne tjene ikke vor HErre lEsu Vhristo , men deres egen Bua ; og formedelst sede Ord og smigrende Tale forfsre de de Enfoloiaes Hjerter. 19. Tht eders Lydighed er bleven Alle bekjendt , derfor glader jeg mia over eder ; men zeg vil , at I skulle vcere <b>vise</b> til det Gode , men eenfoldiae t < l det Onde . 2 tt . Men Fredens Gud skal snart knuse Satanas under eders Fsdder . Vor HErres lEsu Christi Naade verre med eder ! Amen . 21. Timotheus , min Medarbeider , og Lucius , og Jason , og Sosipater , mine Frcrnder, hilse eder . 22. Jeg Tertius , som har nedskrevet dette Brev , hilser eder i HErren . 23. Cajus , min og den hele Menigheds Vert , hilser eder . Grastus, Stabens Rentemefter , hilser eder , og Brodere » Qvartus . 24. Vor HErres lEsu Christi Naade verre med eder alle ! Amen . 25. Men han , som er mcegtig til at styrke eder , efter mit Evangelium og lEsu Christi Prcrdtken , efter Aabenbaringen af den Hemmelighet » , som var fortiet fra evige Lider , 26. men er nu aabenbaret , og ifslge de prophetifle Skrifter , efter den evige Guds Befaling , kundgjort for alle Hedninger til Troens Lydlgbed, 27. ham , den ene <b>vise</b> Gud , vcere A3re ved lEsum Christum i al Evlghed ! Amen .

<h4>1827</h4>stetter sig meget mere aldeles til denne , staaer og falder med den . Det er netop af det evig bestaaende og uforanderlige " Korsets Ord , " den , under Guds Aands Virkning , udvikler sig ; det er af dette Ord , Aanden tåger hvad han forklarer for de Troende alt efter deres aandelige Forstands-Evne , og det er derfor ogsaa saa langt fra at vcere en Biisdom , der er sceregen for de christelige Boglcerde , Skriftkloge og saakaldte lyse Hoveder , at disse meget mere , trods al deres <b>Kundskab</b> og Indsigt , kunne mangle den , medens den simpleste lcege Mand eller <b>Quinde</b> kan vcere i Besiddelse deraf og i denne Henseende gjere mangen lcerd og oplyst Theolog til Skamme . — Ved " denne Verdens Overster " forstaaes ncermest dem , der staae over Mcengden i menneskelig Viisdom og Indsigt , ere dens Lcerere og Ledere , saasom " Pharisceerne og Skriftkloge " hos Isderne ( V . 8. ) , de bersmte Viismcend ( Philosopher ) blandt Hedningerne . Dog er Begrebet af de i borgerlig Henseende Ophsiede ikke udelukket ( 1,26 . ) . Verdens Hsihed i det Hele , forsaavidt den befjeles af Verdens <b>Aand</b> , danner en Modsaetning mod den guddommelige Viisdom i det Evangelium , der kun er tilgjcengelig for de Ydmyge af Hjertet , og Jordens Moegtige , der jo i Ulmindelighed hore til de " Dannede , " de " Oplyste , " ere efter Regelen paavirkede og indtagne af Verdens falske Viisdom . Hos Isderne var det jo ogsaa saa , at deres Skriftkloge tillige dannede deres Ovrighed . Om "Bestcemmelsen" af denne Verdens Overster see 1 , 19. 20. 27. 28. Forsaavidt der blandt de jordiske <b>Vise</b> maatte findes sande Philosopher d . e . Mennesker , som i Fslelsen a ^ at de savnede den Viisdom , der loser Livets Gaader , ssgte den med Lcengsel , da vilde hos Saadanne netop denne deres Ssgen tjene til at berede Guds Viisdom Vei .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112101017'target=_>Ahlfeld, Friedrich, 1866, Menneskelivet i Guds Ords Lys</a><h4>306</h4>Drenge og <b>Piger</b> deriblandt , som intet Hjerte have for sine Lcerere , som ikke lcenger barnligt hengive sig til dem , men i det Hpieste indtage en klog og beregnet Stilling ligeoverfor dem . I deres Ncervcerelfe <b>vise</b> de dem vel en nsdtprftig Lydighed ; de have vel endnu nogen Frygt i sig for Lovens Bogstav ; men der er ingen hjertelig Lydighed . Den barnlige 2 Erefrygt , som Bprn skylde sin Lcerer , fattes hos kun altfor Mange . Af lErefrygten og Mmygheden bor der opstaae en übetinget Lydighed hos Barnet . Lammene fplge Hyrden . Ogsaa her gjcelder , hvad skrevet staaer :  IBprn , adlyder Eders Forceldre i Herren ! thi dette er ret . " Lcereren udpver nemlig en Dccl af Foreldrenes Embede . Og for at lcegge alle Bk < rn dette ret kraftigt paa Hjerte , siges der i Brevet til Ebrceerne :  Adlyder Eders Veiledere , og vcerer dem lydige — thi de vaage over Eders Sjele som de , der stulle gjpre Regnfkab — at de kunne gjsre det med Glcede og ikke sukkende ! thi dette er Eder ikke gavnligt . " Enhver Ulydighed mod Lcereren bpr Forceldrene betragte som begaaet mod dem selv . En Fplge af at der hos Bprnene nuomstunder ofte er Mangel paa hine to Grundb eting elser er , at det ogsaa mangler dem paa de fprste barnlige Dyder i Forholdet til Lcereren , nemlig Kjcerlighed og Taknemmelighed. Intet Menneske kan skjcenke os noget Stprre end den ffjcenker os , som gjor os fortrolige med Herren , som med Kjcerlighed , Alvor og Bsn veileder os paa Frelsens Vei , som fortscetter vor <b>Uddannelse</b> til Lighed med vor Herre Jesus Christus , som fprer os ncermere hen til Ham , der er Hovedet , nemlig Christus . Isandhed , et saadant Arbeide , et saadant Hjerte- og Sjelearbeide er al Tak vcerdt . Med en Lcerer er man ikke fcerdig , naar man har betalt ham Skolepengene. Man kan ikke betale en Lcerer som en Haandvcerker . For Hjertearbeids tilkommer ham en Hjertetak . Paa en ryrende Maade have Hedningerne ofte viist fig taknemmelige mod sine Lcerere . Den romerske Keiser Markus Aurelius overMe ei alene sine Lcerere med Rigdomme og Vcerdigheder : efter deres Dpd satte han . Hedningen , endog deres Billeder blandt sine Huusguder . Dette var just ikke saa urigtigt gjort af ham , som en Hedning ; thi i Sammenligning med de andre Guder skyldte han ialfald sine Lcerere langt Mere end disse . Den gamle romerske Viismand Seneka ' s Disciple bragte ham en Dag Gaver . Nu var der En iblandt dem , som var saa fattig , at han ikke kunde bringe ham Noget . Denne traadte da grcedende fremfor sin Lcerer og sagde :  leg eier Intet ; men jeg bringer oig mig selv , mit Hjerte og min Kjcerlighed . " Og nu christne Born , christne Disciple ! De have jo dog ialfald annammet Mere af gudfrygtige Lcerere end hine Hedninger af

<h4>378</h4>de bedsteVidnesbyrd ; han ( hun ) er stedse bleven Nr . 1 ved Omflytningen . " la , man har endog gaaet saa vidt at maale dem med den samme Alen ved Konftrmationen . Ogsaa der har man omtalt deres Flid , deres Vidnesbyrd , deres fyrste Plads ved Alteret .  Men , " sige I ,  fkal man da ikke rose et Barn ? Skal man ikke yde det sin Anerkj end else , naar det har gjort sig Flid ? " Kjcere Medchristne ! Roes og Mre stal i det Hpieste vcere et Krydderi paa Maden , men aldrig Maden selv . Ve det Barn , hvis Drift og Kraft beroer paa Andres Roes ! Ve det Barn , der har L^ftestcengerne for sin barnlige Pligtopfyldelse udenfor sig selv og navnlig her paa Jorden ! Et saadant Barn bliver et usselt , elendigt , ulykkeligt Barn .  Men , " spsrger du endvidere ,  er da ikke 2 Ere og Roes et godt Middel mod Dovenskab , Uorden og deslige Udyder hos Barnet ? " Jo , ganske vist et Middel . Ofte opfylder det ogsaa Hensigten . Men det er dog intet godt Middel , og Hensigten ovnaaes ogsaa kun overfladisk og tilsyneladende . Naar man om Vaaren vånder Frugttrceer med varmt Vand , skyde de tidligere Knupper , faae tidligere Blade og Blomster og frembringe maaskee ogsaa tidligere nogle tarvelige Frugter ; men de dye siden ud . Der gives ganske vist bedre Midler . At spre Barnet grundigt md i Herren , at forestille det , at Skolegangen og Tilegnelsen af de <b>Kundskaber</b> , som derunder bibringes , er et Guds Kald , en Guds Tjeneste , at bede med det om et lcervilligt Hjerte , om Lust , Flid og Iver , at holde det til at bekjende sin Synd for Gud og bede Ham om Tilgivelse , saa ofte det ikke har viist den behørige Flid og Orden , det er et bedre Middel . Med AZrens yndede Middel driver man maaskee een Djcevel ud ; men saa driver man ganske vist ogsaa en anden md , og det en , som meget vanskeligt lader sig uddrive igjen , som omslynger Barnets hele Hjerte og i alle dets Dage bliver boende i det som en Fiende af dets Frelfe . Dyrebare Lcesere ! naar man trcekker en Plantes <b>Moder</b> op af deres Skjul i Jordens Skjpd og udscetter dem for Solen , saa visner Planten . Saaledes maae ogsaa vore Livsrpdder forblive skjulte i Guds Dyb . Fra Ham maae de trcekke den tilkommende Verdens Kroefter til sig , hvorved det hele Menneske opholdes . Men den , som nceres , drives og bceres af 2 Sren , han har draget sine Livsrpdder ud af Gud . De ligge i Luften , i Solen . Han spger sin Styrke i Mennesker. Hans <b>Sjel</b> hcenger ikke lcenger ved Guds Ansigt . Hans Styrke er ikke lcrnger Herren , men Andres Omdømme . Han har gjort et meget oaarligt Bytte . Hos ham kan der ikke udvikle sig nogen stille <b>Aand</b> , noget dybt Sind og Vcesen ; thi hans <b>Sjel</b> vandrer paa Overfladen og bevceger

<h4>623</h4>saa sog at erholde en gudfrygtig Mester eller Herre . Den Luft , du hele Dagen bevceger dig i , er as stor Indflydelfe paa din Helbred . Hvorsomhelst du kommer hen , saa se til at finde dig et Par gudfrygtige Omgangsvenner , med hvilke du kan samtale om Guds Ord og din Frelse . Hvor langt du end kommer bort fra dit Fcedrenehjem , saa skriv flittigt hjem . Ethvert Brev er en Samtale med dine Forceldre . I ethvert Brev maa du paa en <b>vis</b> Maade trcede srem for dem . Derfor maa du ogsaa gaae paa Raad med dig selv om hvorledes du skal trcede frem for dem . Men for Alt , hvad du gjor , vogt dig for al Logn i dine Vreve ! EtLognbrev bringer dig mindst hunored Mile lcenger bort fra dine Forceldre , uden at du behover at gaae ud af Byen . Ia skriv ret flittigt hjem . Glem ikke det fjerde Bud , fordi om du er paa et fremmed Sted . Din Frelser tog det med sig lige op paa Korset ; tag derfor ogsaa du det med dig til det sidste Sted , du scetter din Fod paa . Gjor dig ingen Bekymring for den dermed forbundne Udgift ; alle Forceldre betale gjerne nogle Skillinger for et Brev fra sin Son . Har du imidlertid en syg Fader eller en gammel <b>Moder</b> , som har vanskeligt for at tilveiebringe Brevpengene , faa betal dem felv . Sult heller for en Gangs Skyld . Naar Postbudet bringer din <b>Moder</b> Brevet , seer hun allerede af Frimcerket , at du endnu er en tro og kjcerlig Son . Arbeid derhos ufortrodent . Ilyv ikke fra den ene By eller Mester til den anden . Der groer ikke Grces paa den Steen , som ofte vceltes . En Svend , som opslider sin Fortjeneste i Stovlesaaler, lcerer Intet . Vcer ydmyg ; thi kun Idmygheden lcerer Noget . Hoffcerdigheden kan Alt og dog tilsidst — Intet . Tcenk endelig stedse paa at det egentlige <b>Maal</b> for din Vandring er den salige Evighed , og hvorledes du skal komme did , det lcerer Ingen dig saa sikkert som dit Nytestament . Ligesom vi , mine elskede Lcesere , have vceret med den unge Mand paa hans Reiser og Vandringer , saaledes ville vi ogsaa vcere tilstede , naar han efter flere Aars Forlov kommer tilbage til sine Forceldres Huus . Det er en vigtig Dag . Hvor opmcerksomt betragte ikke hans Fader og <b>Moder</b> ham ! Hvor utaalmodigt vente de ikke paa at han rigtigt skal oplade sit Hjerte for dem ! De ville vide , hvao han er bleven til . Og hvad erfare de da ? Der er fire Slags Erfaringer, de isaafald kunne gjore . Een Dccl af de unge Mcend er vandret ud fra Hjemmet uden at tåge Noget med sig . Verden og dens Lyst eiede deres Hjerte , da de drog bort , og de eie det fremdeles , naar de komme tilbage . Deres Hjerte er kun blevet celdre , haardere og fortroligere med Synden . Da staae Frcenderne sorgmodige hos den Hjem-

<h4>679</h4>Herre , styr og led vore unge Mcend og <b>Piger</b> saaledes , at de i Valget af sine Livsledsagere ikke bespprge sig med Kjsd og Blod . Lad dem oplstfte sine Dine til Dig og spsrge Dig , om denne Brud eller Brudgom ogsaa vil hjcelpe dem at holde fast ved sin <b>Sjels</b> Brudgom , om han eller hun ogsaa vil lette dem Fremskridningen paa Frelsens Vei . Og svarer Du dem da i den Hellig-Aand Nei , saa lad det ogsaa vcere Nei , saa lad dem yomyge sin <b>Sjel</b> under dit Udsagn , som det hsieste og sidste Ord . Ak , Herre , bevar dem , saa ikke SkMhed , Gods og Guld , Stand og Herkomst stutte disse Forbindelser , men den faste Overbeviisning , at de begge have een Herre og een Tro , den faste Overbeviisning , at nettop de ville hjcelpe og styrke hinanden paa Veien til det Jerusalem heroventil . Herre , vcer du overalt Trediemand i Pagten ; vcer Du dens <b>Sjel</b> ; veer Du deres sande Baand ; veer Du deres Freds og Glcedes Klippe ; vcer Du det <b>Maal</b> , hvortil de begge vandre . Og byg Dig saa Huse , hvori Du selv kan boe , som ere Guds Pauluner hos Menneskene , som

<h4>695</h4>Grcendserne ; men disse gjsre sig dog senere gjceldende , og Misforholdet virker da forstyrrende gjennem hele Livet . Der bor hetterikke ved din Forlovelse opstaae nogen Klpft mellem dig , dine Forceldre og S ^ stende . Jeg maa derhos lcegge dig paa Hjertet , at man fsr Forlovelfen bM have lcert hinanden grundigt at kjende . Man pleier at sige , at man ikke bpr slutte Venskab med Nogen , fsrend man har fpiist een eller to Skjcepper Salt med ham . Et Mgtestab er noget langt Mere end et Venskab . Fra en Ven kan jeg trcekke mig tilbage , men derimod ikke fra en <b>Hustru</b> eller Mand . Vogt dig desaarsag for en overilet Forlovelse . Jeg har svurgt mange 2 Egtepar , hvis Fred var bortvegen og som slet ikke kunde stemme overeens , hvor lcenge de havde kjendt hinanden , fDr de blev forlovede , og da har jeg faaet til Svar :  Fire Uger , sex Uger , otte Uger . " Og tilsidst kunde de endog sige :  Vi kjendte hinanden i Grunden slet ikke . " Spurgte jeg videre :  Talte I nogensinde med hinanden om Eders Troes Grundvold og Haab , — bad I nogensinde med hinanden , — gik I nogensinde til den hellige Nadverd sammen ? " saa lsd Svaret :  Nei " og atter  Nei " og endnu engang  Nei " . Hvor skal der da komme nogen Enighed fra ? Forlovelser , der sluttes i Kjpdet , af Kjpdets Lyst eller verdslig Beregning , dele alt Kjpds Lod . Alt Kjsd er Grces , og al dets Yndest som Markens Blomst . Unge <b>Piger</b> , som ingen Lyst have til det stille huuslige Liv , som elske Balsalen hsiere end Stuen og Kirken , som med Dina gaae ud sor at see paa Hedningernes Dptre , blive aldrig gode Hustruer . Din Bruds kjcereste Steder bsr vcere Herrens Huus og hendes eget Huus . Kun den <b>Pige</b> byr vcere din Brud , som kan sige til dig som Ruth :  Hvor du gaaer hen , vil jeg gaae , og hvor du forbliver , vil jeg forblive; dit Folk er mit Folk , og din Gud er min Gud / ' Kun med hende kan du vcere <b>vis</b> paa at komme til at leve i Fred og Enighed . At imidlertid en saadan Pagt aldrig lM sluttes uden Foreldrenes Samtykke , at disse endog byr adspsrges for Bruden selv , og at man fprst maa have sikret sig deres Velsignelse , er Noget , hvorom ingen Christen kan vcere i Tvivl . Det Bifald , som ester Forlovelsens Indgaaelse afnpdes dem , er intet Bifald , den Velsignelse , som da gives , ingen Velsignelse . Du bsr agte deres Ord saa tMt , at du ogsaa stille ydmyger dig under deres Nei . Og vil du <b>vise</b> dem denne 2 Ere , saa bsr du allerede adspmge dem , naar den fyrste TilvMelighed fremspirer i dit Hjerte , og forinden du endnu har gjort nogetsomhelst utvetydigt Skridt henimod <b>Maalet</b> . Til disse fsrste og vigtigste Fordringer komme endnu nogle andre . Wgtefolk bsr ogsaa med Hen-

<h4>931</h4>for sine Slcegtninger og Bekjendte . Naar Herren aabenbarer sin Herlighed paa dem , naar Han gM dem salige midt i deres Elendighed , naar Han giver dem Hovedpryd istedetfor Aske , Gloedes Olie istedetfor Sorg , Lovpriisnings Klcedebon istedetfor en vansmcegtende <b>Aand</b> , naar de staae der som Retfcrrdighedens Terebinther , Herrens Plantning til Hans Forherligelse , naar den Halte springer som Hjorten, og den Stummes Tunge jubler , o , hvor prcediker da ikke denne den Hellig-Aands Naadevirkning ! o , hvor blive da ikke Herrens Gjerninger aadenbarede ! De kunne imidlertid ogsaa prcrdike og vidne paa andre Maader i sine Familier. De kunne ststte Huset med sine Bpnner og frembcere dets Ve og Vel for Herren . Ved Hjcelp af egen dyb Erfaring kunne de <b>vise</b> Elskende , Broder- og Sssterbsrn saavelsom Andre Salighedsveien . De kunne forkynde sin hele Fremde- og Vennekreds Guds underfulde Gjerninger . Her i Byen ( Leipzig ) levede for ikke lcenge siden en gammel blind <b>Kone</b> , som i Vantroens MiZrke blev en Vederkvegelse for mange tørstige Ejele . Hun kunde vidne ganske anderledes om det Lys , som har skinnet i Merket , end de fleste Mennesker med klare Dine . I Aaret 1739 sendte Brsdremenigheden to Missionoerer til Vestindien . Den Ene , Albin Theodor Feders , havde studeret ved tre Universiteter og var en dygtig Theolog , som man gjorde sig store Forventninger om . Den Anden , Christian Gottlieb Israels , var en halt og pukkelrygget Skrceddersvend . Den stakkels Karl havde et ynkeligt Udvortes ; men med hans Indre stod det herligt til . Ter var han retvoxen som en Gran , strakte sig i fast Tro og barnlig Kjcerlighed opad til sin Herre og havde intet andet Ynske end at tjene Ham blandt de arme Negre . At han selv formaaede Noget , var han langt fra at indbilde sig ; han kjendte sin egen Ekrpbelighed ; men Herren skulde vcere stcerk i ham . <b>Maalet</b> for deres Reise var Ven St . Thomas . Paa et dansk Fartpi reiste de til St . Eustatius , hvor de gik ombord paa et engelsk Barkskib , som skulde bringe dem til deres Bestemmelsessted . De led imidlertid Etibbrud paa Klipperen Skrov . Størstedelen af Mandskabet reddede sig i Storbanden og kom iland paa Ven , hvorimod de to Missioncerer maatte blive tilbage paa Etibet . Fe < rend dette gik under , harde de sundet et midler ? tidigt Tilflugtssted paa et Udet Skicer . Feders forsagte at naae Land paa en Ncekke Stene , som ragede op over Våndet, men gled og fandt sin Dgd i Velgerne . Den halte Israels derimod blev omsider trukken iland paa Ven ved Hjcelp af et ndkastet Toug og saaleoes reddet . Da tcrnkte Mange :  Ten dygtige Arbeider ligger paa Havsens Bund ,

<h4>1107</h4>ryfte dit Kors ' s Frugter ned i deres Skjod . la , velsign vort Kors for os . Du naadefulde og salige Korsdrager ! Amen . Mine i Herren elskede Lcesere ! Der er ingen Stand og Stilling , som Mennesket stunder saa meget ester som 2Egtestanden og en egen Familie . En stor Dccl as Ungdommens Mmc og Arbeide sigter til dette <b>Maal</b> . Der er Intet , som i Mere Grad beskjeftiger den unge Mands og <b>Piges</b> <b>Sjel</b> end de Forestillinger , de gjore sig om dette nye Afsnit as sit Lw . Der er neppe Nogen , som ikke efter beoste Evne har udmalt sig det med de deiligste Farver . Og dog er der ingen Stand , som bcerer en dybere Sorg og Smerte i sit Skjod end nettop ZEgtestanden . I mange 2 Egteskaber begynder Kummeren allerede kort efter Bryllupsdagen . Solen er neppe stegen op , forend der legger sig morke Skyer foran den . Gledens Taarer paa Bryllupsdagen efterfolges ofte meget snart af Sorgens Taarer , Perlerne i Krandsen og paa Halsen af Perlerne paa Kinderne . Og ofte er Wgteskabet en Korsets Gang fra Alteret til Graven . Hvo vil maale al den Hjertesorg og telle alle de Taarer , som Wgtestanden har fremkaldt ? Lader os forst betragte det Huuskors , som Gud Herren paalegger Mgtefellerne selv ! Ikke sjelden er det Tilfeldet , at den Ene af dem allerede bliver syg eller begynder at skrante nogen Tid efter Brylluppet . Mange Mend og Koner har jeg hort klage og sige :  Det var kun en kort Tid , jeg havde min <b>Kone</b> , min Mand frisk ; fra det forste eller andet Aar af har min Wgtestand veret en Lidelsens Skole , en stadig Sygepleie . " Eller Fattigdommen slutter sig allerede tidligt til det unge Par som en uadskillelig Folgesvend . Eller Herrens Haand hviler tungt paa Nornene . Tidligt tåger Han dem bort , eller Sygdom bliver deres Lod , saa at hverken Barnene kunne glede sig rigtigt i sit unge Liv eller Foreldrene i Bornene . O kjere Leser ! gaaer det ogsaa En og Anden af Eder saaledes , vide ogsaa I at tale om et Morke , som nesten fra Begyndelsen af har ruget over Eders Samliv , og om andre Perler end dem , som Bruden var smykket med paa Bryllupsdagen , saa seer vel til , at I ikke berove Eder selv Korsets Velsignelse ! Al Trengsel er en haard og suur Skal , hvori Gud har indesluttet en kostelig og sod Frugt . Al Revselse synes , medens den er nerverende , ikke at vere til Glede , men til Bedrovelse; men siden giver den igjen dem , som ved den ere svede , Retferdigheds salige Frugt . Jesus Christus , som har gaaet under Korset og udkaaret det til sin Fane , vil under Korset drage ind til Eder . I det nye Testament seer du ret tydeligt, hvorledes Trengselen maa aabne Doren for Ham .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010021803052'target=_>Zschokke, Heinrich, 1857, Huusandagtsbog</a><h4>511</h4>Uugdom udeu saud <b>kvindelig</b> <b>Dannelse</b> , ofte endog med fordervet <b>Aand</b> og Hjerte , indtreder i det egtestabelige Livs vanskelige og vigtige Forhold ? O , min christelige Veninde , som snart sial leirete Mgtestabets Dorterstel , hust paa , at det ikke blot er ^ in tilkommende Mand , hans Stand og Formue , som du har at takke for din tilkommende Lykfalighed , men ligesaameget de gode Egenskaber , du som ung <b>Pige</b> har erhvervet dig . Husi paa , at disse Dyder ere deu bedste Morgeugave , du engang kan bringe din Brudgom . Betenk , at du kan miste al anden Lykke men aldrig den , der er grundet paa din egen Kraft og Overbeviisning . Derfor stal on allerede nu ove dig i huuslige Forhold , som den unge Mand i Skolen eller Verkstedet , at du kan blive den Verdigste af dit Kjon . Erhverv dig en Prydelse , der ikke taber sig med Aarene , aldrig gaaer af Mode , men gjor dig endnu som bedaget Matrone verdig til Agtelse .

<h4>1012</h4>Den Fattige er ikke saa meget at beklage , fordi han maa noies med daarlig Kost og klede sig i Pjalter . Der gives Folk , blandt hvilke de , vi kalde fattige , vilde blive ansete for misuudelsesverdig rige . Men han er at beklage , nåar han ogsaa mangler dette Lidet og folgelig ikke kan sorge for Legemets Sundhet», eller nåar han for at fortjene faa Meget maa anvende al sin Tid og Flid uden at kunne faae nogen tilovers til at sorge for <b>Sjelens</b> Sundhed , og Mennesket altsaa synker ned til at blive et tankelost Treldyr eller en dod Mastine . I begge Tilfelde forfeiles <b>Maalet</b> for hans Tilverelse . Det er fra dette Synspunkt , man maa bedomme saavel enkelte Personers som hele Folkeslags Fattigdom . Gt - Folk , som lever paa en karrig og nfrugtbar Jordbund og neppe kan friste Livet , og som fra Morgen til Aften maa anstrenge sig for at tilfredsstille sine forste Fornodenheder , er virkelig fattigt ; det bliver aandsfattigt , tregt forn det udhungrede Dyr , eller vildt og krigslystent , som det hungrige Rovdyr , der endnu er sig sin Kraft bevidst . Hvad der her er sagt om Folkeslag , gjelder ogsaa om enkelte Familier . Der gives saadanne , fra hvilke der udgaae enten syge Betlere eller grumme Rovere . Selv i saakaldte rige Steder kan der herske Armod trods den travle Virksomhed og gode Fortjeneste , og de store Udgifter , der nu ere blevne en Nodvendighed , kunne vere saa trykkende , at mange Mennesker , der med mere Noisomhed vilde tuune have Overflod , nu neppe engang vove at indtrede i den hellige Wgtestand af Frygt for ikke at kunne ernere <b>Hustru</b> og Born . Saaledes er ogsaa Landbostanden i Virkeligheden fattig , om den end kan glede sig ved en frugtbar Jord og en rig Host , nåar

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012073108081'target=_>Hauge, Hans Nielsen, 1867, Christelige Lære</a><h4>124</h4>imod . Eller cre I ndgangne at se et Menneske , i blodo Kladcr ? se ! de der bcercr blode Klceder eve i Kongerc- Huse . Vil de se Verdens Hcrligbcd , alle glimrende Ting , Tolv og Guld , med Silke , Floicl , sint Liu.ncd , Skarlagcn med merc , saadanne blode Klcedcr havcr de , ccr cre i Kongers Huse ; ligesaa ere de ikleedtc blode Klccrer eller deres Sind , det er blodt ; tbi de frvgter for , at de stnlde miste noget as deres verdslige Wre , om de stnlde sige Sandbed og ilke an , e Pcrsoner, eller at de stnlde straffes , bvorfor de beller bvller ; ,aadanne er ilke lobannes og bans Lige lig ; tbi de bolder stg Me i Kongers . hnse , men til de Fattige og Ringe ; ttn re tragte ingen verdslig Hoihed ester vaa Jorden ; lommer nogcn de Verdens Hoie til dcm , saa sige de Sandbed . Eller bvad ere I ndgangne at se ? Ville I se en Propbet ? ja jeg siger Eder : mange Propheter vidnede Sandhed og talede det iom Herren sagde dem , hvorover de vleve as Sandheds Fiender wnaflede og pinet , ligesom lobannes nn er låstet i Fengsel for ban ' sagde Herodes Sandhed . Men jeg siger Eder : at han , nemlig Johannes , er langt mcre end en Propbet , for de vare fodt i Synd og siden salvet med den Helligaand til at vropbetere , men denne var fyldt med den Helligaand as Moders Liv , og lydig til at ndrette Herrens Gjerning som en Engel ; thi denne ' er det som er skrevet om : jeg sender mm Engel for dit Anfigt , som skal bercde Vei sor dig . I Vers ester vor Text , siger lesns , at ingen af dem , som er fodt af <b>Kvindcr</b> , er storre end lobannes ; tbi endskjont ban er fsdt af en <b>Kvinde</b> , faa bar ban dog den Helligaands boie Gåver , som renser og bcgaver bam , han gaar for mit Anfigt af mit Lys , og nn bereder han Vei , det Onde , raader Eder til Omvendelsc , og oplyser af mit Lys , at de som sidde i Morket tan nu se et stort Lys . Esaias 9 Cap . 2 V . Vil de nu ved hans <b>Aand</b> lade rydde Vei , saa vil jeg indkomme med min Herligbed , vandre i Eder , og fylde Eder med Naade , <b>Visdom</b> og Kraft .

<h4>161</h4>Folk , som fla ! stabcs , stal frygte Herren , Psal . 102 , 9 V . Disse Davids Ord ere at betcenke , hvorfor Herren har ladet sine Tjencre strevet sit Ord ; thi det er fleet og steer for at stade eller igjenfore Menneflene til at frygte og elste Gud , som vi lcerer i hvcrt Bud , og det siger Paulus i Ebr . 2 ( sav . , at det bor os dcsmcrc , at tåge vare paa de Ting vi bave bort , at vi ikle stal forgaa ; thi er det Ord bleven fast som er talt formedelst Englene , og hver Overtrceder og Ulydig fit sin rette Lon , hvorledes skulde vi da undfly , dersom vi ikle agte saa stor en Saliggjorelse , hvilken , ester han tog sin Begvnvelse at tale ved Herren , er bleven stadfestet for os , formedelst de som have hort ham , i det Gud vidnede med baade Tcgn og underlige Gjerninger , ester den Helligaands Uddelelse ester b.ins egen Villie . Og i vor Teztes Ord , siger Apostelen : at Gud i fordum Tid haver talet mange Gange og i mange Maadcr til Fedrenc ved Prophcterne ; thi Gud truede vore forste Forccldre med Dodens Straf , dersom de ovcrtraadte Bndct ; og da det ste.de , saa kom Faldet , hvorfra han lovcde dem , Forlosning i <b>Kvindcns</b> Sced , 1 Mose B . 3 Cap . 15 V . og forelagde dem de Trcengsler , de stnlde lide , og hvorledes de stnlde stride mod Synden , der havoe attråa til dem ; men de skulde hcrste over den . 4 Cap . 7 V . Dette trettedc hans Aano med dem , ved alle sine Prophcter , med niauge Straffe og store Gjerninger , baade om Befriche og Odeleggelsc , at frelse og fore til Hcrlighed de som vilde lyde , og udstctte af lordcu de Gjenstridige . Da han nu til siu Wre ved sin <b>Aand</b> og Tjencre havde overbevist baade om Straf og Frelse ved sit gnddommelige Ord , saa taler han i de sidste Dage vesentlig ved Sonncn , hvilken han har sendt at treve Frngtcn as sin Vingaarv , at vi stnlde givc ham 3 Ere og tjene ham alenc ; thi han er Arvingen , Math . 21 Cap . 37 , 38 og ved ham , ere alle Ting gjorte ; thi det er det Ord , som Johannes siger : Der er ikke et Stykke gjort , af alt det som gjort er , nden ved ham , Joh . 1 C . han , nemlig Jesus , Guds Son , han er Herlighedcns klare Skin , i komme Hcllighed , som ingen kunde overbevise om nogen Synd , Joh . 8 Cap . 46 V . og Guds Verclses rette Billede ; tbi hvo der saa ham , han har og seet Faderen , 14 Cap . 9 V . og han tjendle hans Hjertelag og Verclse , som er Kjoerlighed ; thi ban er Kjcerligheden sclv , dens Vei , <b>Maal</b> og Fylde , som bel

<h4>252</h4>Men da , Tidens Fylde kom , sendte Gud sin Son fodt af en <b>Kvinde</b> , som blcv lagt nnder Loven , at vi skulde faa sonlig Udtaarelse . Heri ser vi Gnds Forjattelser , at <b>Kvindens</b> Sad stal sonderknnse Slangens Hoved , og som <b>Kvinden</b> blev forraadt af Slangen , til at overtrcede Guds Bud , tog af det forbudne Troe og aad sig Doden . Dette , nemlig Doden , er Naturen Fiende til ; thi Naturen elster Livet , ja vil leve efter sit eget Behag , der strider imod Guds Villie og Lov ; nu vilde Gud igjen forlese det faldne menneskelige Kjon , ei alene fra den aandelige Dod og timelige Trceldom , af Sorg og Ufornoielighed i Verden , men og fra den evige Jammer , som denne Verdens Sorg og Kjcerlighed forer efter sig ; han vilde ei alene fore os til et fornyct Sind , <b>Visdom</b> og Magt at bruge de flabte Ting , leve fri i Guds Kjarlighed , at nyde hans Gåver til vort Legems Styrke , saa vor <b>Sjel</b> derved tan prise i Tiden ; men og berede os at indgaa til det evige Liv , ! . g der at arve det Rige , som var forud bcred , eller af Gud ~ , ' iittet til sin Son og hans dyrctjobte . Giendoms Stat ; nu urlcndte ban ban : ved sin Aando Undfcmgcise oa fod af en l

<h4>366</h4>I vor Dages Evangelie lcefer vi , at der Jesus var tolv Aar gammel , gik han med sin <b>Moder</b> og Joseph op til Jerusalem efter Hoitidens Scedvane ; og der de havde endt disse Dage , gik de igjen hjem , og Jesus blev i Jerusalem , og Joseph og hans <b>Moder</b> vidste det ilke , men de mente han havde vceret iblandt Selftabet ; de lom frem en Dages Reife , og lcdte efter ham ivlandt Slcegtninger , men de fandt ham ikle . IsderneS Kirketjeneste var bleven til en Scedvane , og deres Dage vare bestemte ; hvilket Gud havde ' maattet foreskrive dem ; for de vare uordentlige og uvillige , at de maatte tvinges , og de fleste af dem som tog det Ndvortcs iagt , stolede derpaa , og alligevel levede i Synden , bar ingen Frugter deraf , men mente , nåar det Udvortes var gjort , saa var det nok ; de Lcerde , Mperste og Fornemste havde deraf sine Indkomster; af Ondstabs Skalkhed tvang de den Ringere , overtalede den Vandkundige til at tro det var opfyldt alt , nåar de havde offret og gjort den udvortes Gudsdyrkelsc , indbildte dem at det var nok til Guds A3re , endfljont Herren siger : Hvad stal jeg med Eders mangfoldigc Offere ? jeg er mcet af Brcendoffer og de fede Kvceg , min <b>Sjel</b> hader Eders Nymaaner og Eders bestemte Heltider , jeg er kjed af dem ; nåar I end udbrede Eders Hcender , da vil ftg stjule mine Wine for Eder ; thi Eders Hcender ere fulde af Blod . Es . IC . Men Jesu <b>Visdom</b> var af guddommelig Art , at endfljont han var et Barn , saa var han dog mcegtig , og hans <b>Lcerdom</b> retstaffen til at soge sin Faders Wre ; ban vilde da soge de Lærdes Skare , og vcrre noget iblandt dem ; dette vidste ikle hans <b>Moder</b> og Joseph , men de mente han var iblandt sin Slcegt

<h4>746</h4>kunde Kau ikte lagge nogen Priis til dem , fordi de havde ikke Magt til at <b>vise</b> Fienden nbgen prisvardig Gjerning ; og laste dcm vilde han mindre ; thi han havde den Kjerlighed , at han ikte vilde af Had bebreide nogen Fiende , mindre sine oprigtige Discivle ; og Judas som var falfl bebreidede sig sclv . Men om sig selv svarer han : leg haver talt aadenbarligen , for jeg lcerte altid i Skolen , i Tempelet , der som loderne vare forsamlede . leg haver intet talet i Lsndom , hvi sporger du mig ad ? ( thi du troer ikle mig , ) of dertil haver jeg ilte skjult det- spsrg dem som haver hort mig . Saalcdes maae Christi Vidner altid bekjendt , at de soger Samfuud og aabenbarer sine Gjerninger , ' at hver den som gjor godt kan komme til Lyset . Men saa hader Morthedens Born det ; at en for et godt ' Arbeide kan faa en ond Lon , og for et ret Svar Vrcde , Forvittrelse og Slag . Thi som lesns dette talede , gav en af Tjenerne hamen Kindhest og sagde : Skal du saa svare den Npperste Prcest ? Jesus sagde : Haver jeg talet ilve , da beviis at det er ondt ; men haver jeg talet ret , hvi slaar du ' mig ? Her haver vi Exemvel vaa , hvorledes vi stal ilte hadc vor Fiende , men overbevise ham ; thi Christus , som gav ret Svar , vilde heller ilke lade sig forvende eller fortie det , da Findcn vilde forvende Retten ; dette saa dv Onde , og MMtte dog slaae sig selv , at de var de , som straffede det Gode . Imidlertid urens dette flcte , stod Peder i Forgaarden og varmede sfg ; og en anden <b>Pige</b> saae ham , og begyndte atter at sige til dem , som der stode hos , og de til ham : Du er en af hans Discivle ; og han nagtede atter og sagde : leg er ilke , eller jeg kjcnder ilke det Menneske . Noget derefter ved den anden Stund , stadfastede en anden som stod omkring , og sagde : Sandelig, du er en af dem , thi dn er en Galilccer og dit <b>Maal</b> ryber dig . ( En sand Christen bor kjendes vaa sit <b>Maal</b> , ved sin sagtmodige , aandelige , fornodne Tale . ) Men da Peder Horte at saa mange kjeudte ham , og isar den Upverstc Prcestes Svend , hvis Fiende han var og havde hugget Vre af , saa begyndte han at forvande sig ogsvarge : leg kjender det Me nnefte ikle , den I tale om . Her se vi Mennestens Elcndigbed og Fristerens Magt . Ja ikle alenc det Menneske , som gaar i Ondskab med Villie til at gjore det Onde , men og den som gjor godt , dog i Sikkerhed salder fr a en Last til en anden ; og saaledes gjor Fienden nåar han faar et Menneske i Angest

<h4>1019</h4>torre ? Disse Christi Ord kan jeg ikte udfore med saadan Fuldtommenhed , sand Overbeviisning , Advarsel og <b>Lcerdom</b> , som der ligger Grund i dem . Ak ! hvilken Hob folger der ikte lesum og hans Efterfolgere eller Korsbcerere med Klage og Hylen over de Lidende , hvorved de lcegger storre Byrder paa dem ; thi de gjor ikle som Simeon , bcerer Korset , hvilket kunde stedt nåar de opmuntrede de Lidende til Frimodighed , og at vcere taalig i Trcengselen , ved det visse Haab om Overvindelse og Befrielse fra Lidelsen . ; men disse Medfolgere er snart fulde af Sorg , nåar Modgang <b>moder</b> , og i Medgang af letftndig Glcede , som er kjedelige Mennefter , der kan snart vcere mild , og strår , vred , snart glad , fornsiet og strår , sorggiven og misfornoiet , ja er ustadig i alle sine Veie , som Apostelen Jakob 1 C . 8 V . siger , da derimod en sand Christen er stadig . Vi finde og i vore Tider et Slags falske Aander ; som vi og tan erfare af adstillige Bsger , at der har vceret for saadanne Antichristens Aander af Djcevelen forvendte , afviget baade fra Christi <b>Lcerdom</b> meget ; men iscer det sande Christi Levnet , hvilken hyklelfle <b>Aand</b> , falste Sekt og forforiste <b>Lcerdom</b> er den vcerste og farligste Religion i Verden- finde 3 , som er den sineste , hvilke en snarest kan blive bedragen af at fanges i Djevelens Garn og vcerst at komme los fra . Jeg haver selv vceret ncer ved at fanges deri , og tan ikke udtrykke i Ord hvor bedragelig og mange Antichristens Aander der er udgangne i Verden , saa jeg mest maa formane mine Glstelige , at de ikte tror hver <b>Aand</b> , men vrover Aanderne , om de ere af Gud , og beljender lesum Christum at vcere kommen i Kjodet . 1 Joh . 4 C . Dog disse falste Forforere kan nok bekjende lesum med Mund , at han er kommen i Kjod , og beraabe sig paa hans Lidelse og Dod ; men aldrig tror de i sit Hjerte og beviser det i kraftige Kjcerligheds-Gjerninger , at han i dem bor , og de viser hans Frugter , med den fuldkomne Forncegtelse og Ovofrelfe til Herrens Tjeueste , at gjore alle Mennefler godt og elske dem som sig selv , i at forlade sine egne Ting og Gode her i Verden , ligesom Christus forled sit Gode hos sin Fader , for at tjene os , som og vi gjor for at tjene vor Nceste ; thi det ncegter de ; dog nogle kan vel eve lidet uden Acmdens Drift , alene af den Overbeviisning de har , da de ilke er bommen i den sande Kraft og Drivelse af Aanden , om de drages af den : thi Guds <b>Aand</b> kan mange

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012071305102'target=_>, 1870, Nordiske Digtere i vort Aarhundrede</a><h4>6398</h4>Wergelands personlige Stilling var i disse den ordnede Productions Aar saaledes , at den indeholdt Betingelserne for , at hans <b>Aand</b> kunde udfolde sin mærkelige <b>Begavelse</b> sikkert og roligt . Han var kommen ud over sin Ungdoms Gjæring uden at have mistet dens Bevægelighed , han levede med en elsket <b>Hustru</b> i et lykkeligt Hjem og havde med sin store Arbejdsomhed Tid nok fra sin Embedsvirksomhed som Rigsarchivar til at hengive sig til den digteriske Skaben eller sysle med sine kjære Blomster og andre Yndlingstilbøjeligheder . Vel blev ej heller denne Periode af hans Liv fri for offenlig Strid og privat Modgang , og » hans Halvbroder » Siful Sifadda maatte atter tåge sit Kogger frem og sende Farcerne « Engelsk Salt * og « Vinægers Fjeldeventyr « over i Modstandernes Lejr som bevingede Pile ; men hans Liv var i det Hele og Store lykkeligt , han stod i sin Manddoms fulde Kraft paa en vel erhvervet Popularitets faste Grundlag med Lyst til at virke og med et sikkert <b>Maal</b> at sigte efter . Sørgeligt var det derfor , at hans Livsbane skulde afbrydes saa tidligt . I Foraaret 1844 mærkede han de første Spor af den Sygdom , der efter et Aars Forløb lagde ham i Graven . Under dette lange Sygeleje arbejdede Wergeland med en forbavsende Kraft , besørgede anden Udgave af « Skabelsen , Mennesket og Messias « , skrev for Almuen og digtede nogle af sine skjønneste Digte , renere og fuldkomnere , jo nærmere Døden kom . Den endte hans ildfuldt bevægede Liv den 12 Juli 1845. Fem Dage efter blev han fulgt til Jorden af et Følge saa stort , som det aldrig forhen var seet i Christiania .

<h4>11524</h4>Skal denne i Korthed characteriseres , da fæster Øjet sig først og fremmest paa Digterens Hovedevne : den dybe , rene og varme Følelse . I denne bunde hans Fejl som hans Fortrin ; hvor den endnu flyver hjemløs omkring mod de übestemte Længsler , der blive dens Omrids taagede og dens Udtryk klynkende som i flere af Munchs tidligere Digte ; hvor den veed sit <b>Maal</b> og møder sin Gjenstand , der omfatter den den med en mild og kvægende Varme , som udgaaer fra Digterens inderste Natur , og der udtaler den sig i et ædelbaarent og oprindeligt Sprog , et fint og træffende Udtryk for Stemningen . Hvis man ikke vil misforstaae Ordet , kan Munchs Digternatur betegnes som en <b>kvindelig</b> Natur : den har Kvindehjertets Medfølelse med Alt , hvad der lider og sørger , den har <b>Kvindens</b> dybe Forstaaelse af Naturlivets elegiske Side , den har det fine melancholske Slør , som saa ofte er draget over et <b>kvindeligt</b> Aasyn , og som en raa Betragtning strax er rede til at stemple som » Sygelighed « . Skjønnest aabenbare disse Egenskaber ved Munchs Poesi sig i de senest skrevne af « Digte , gamle og nye « og i de « Nye Digte « saa vel som i den ypperlige Romancecyclus « Kongedatterens Brudefærd « , lige som i det Hele tåget Munchs lyriske <b>Begavelse</b> i Romancen har fundet en Form , der slutter godt om den . Mere til — bagetrukne virke de i den fortræffelige Samling « Billeder fra Nord og Syd « , hvis Prosaform har fremkaldt en højere Grad af Objectivering , end der ellers er Munch ejendommelig .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014121208040'target=_>Ahlfeld, Friedrich, 1878, Menneskelivet i Guds Ords Lys</a><h4>301</h4>maa ikke tages saa ganfle bogstaveligt " , saa siger Barnet sig selv det Samme ved ti andre Skriftsteder ; thi Tvivlen sidder i ethvert Menneskehjerte . Indplant tvertimod Wrefrygten for Skriftens Ord ret fast i den unge <b>Sjel</b> . Den maa staae fast ; thi man kan vcere <b>vis</b> paa at der vil komme Storme i Livet , hvorved Troens Trce udscettes fur at oprykkes med Rode . Naar du derfor vil fortcelle , saa tag dig ikke Sagen for let . Du har i dine yngre Aar lcest og lcert Bibelhistorien . Imidlertid er Mangt og Meget faldet ud as din Hukommelse eller staaer ialfald mindre klart for dig ; men du troer desuagtet , at du nok stal kunne fortcelle det . Du vil fortcelle det saa godt eller saa flet , som du kan . Du bekymrer dig ikke om at Sagen her eller der faaer en anden Vending eller Farve . Men det bor den ikke ; dette er falskt og fordcerveligt . Gjennemlces forst selv , hvad du vil fortcelle, saa at du kan gjengive den rette Ting med de rette Udtryk . Fortceller du idag saa og imorgen saa hvilket let skeer , naar vor egen <b>Kundskab</b> er upaalidelig saa vil Varnet , som holder fast ved den forste Fortcelling snart sige dig lige i Vinene : „ Det er ikke sandt , Far ( <b>Mor</b> ) / ' Du maa fortcelle den ene Gang som den anden . Men bed fremfor Alt Herren , at Han vil lade sin Sol opgaae og sit Regn og sin Dugg falde paa de Scedekorn , du udstroer i den unge Jord , saa de maae spire og fceste Rod og voxe til vor hoilovede Guds ag Frelsers 3 Ere . Da vil den tid ' lige Underviisning i Herrens Ord ogsaa bcere sin Velsignelse . Hvilken da ? Tcenk endnu engang tilbage paa vor Text . Tidligt havde Eunike og Lois underviist Timotheus i Guds Ord . At Varnet havde fundet Glceds deri , og at denne Underviisning havde baaret sine Frugter under Opdragelsen , derom er ingen Tvivl . Men den storste Velsignelse viste sig forst senere . Da Timotheus var bleven en Ingling , , kom Paulns med Barnabas til Lystra , den By , hvor hans Forceldre boede . Paulus prcedikede der den Herre , om hvem det gamle Testament havde svaaet i Loven og Profeterne. Da saldt det som Skjcel fra Timotheus ' s Vine , og af Forbillederne og Spaadommene forstod han , at dette var den Frelser , han tilligemed sin <b>Moder</b> og Bedstemoder havde stundet efter . Hans kjcere Arv fra Ungdommen blev ham en Tor til den evige Arv . Aldrig , aldrig fornegtede denne Timotheus siden Herren . Medens Demas og Andre faldt fra og fattede Kjcerlighed til Verden , forblev han Herren og sin aaudelige Fader Paulus tro . Dette havde for en stor Deel sin Grund i hiin tidlige Underviisning . Ordet og Forjcettelserne stod for faste hos ham , til at han nogensinde stulde have kunnet lssrive sig fra den Forjcettede . Og

<h4>357</h4>Forældrenes Medhjcelpere ; de tage en Deel af deres Kald paa sig . Hvad Forceldrene ikke kunne yde , det yde Lcererne som Apostelen Peter siger : „ Efter som Enhver af Eder har faaet en Naadegcwe , saa tjener hverandre dermed , som gode Huusholdere over Guds mangeslags Naade ! " Ja , i den chriftne Kirke indtage Lcererne en ganske anden Stilling end i Hedningeverdenen og blandt Jødefolket . Om Skoler for Ungdommen er der ncesten slet ikke Tale i det gamle Testaments Boger . Kun visse Profeter samlede fra Samuels Dage Skarer af unge Mcend omkring sig for at uddanne dem til den profetiske Tjeneste . De underviste dem i Loven , med hvilken Folkets Synder skulde straffes , men over hvilken tillige Spaadommen om Opfyldelsens og Naadens Dage hvilede . Til Profeter kunde de vistnok ikke opdrage dem . Den hellige Spaadomsevne var en fri Gave af den Hellig- <b>Aand</b> . Men til sine Redskaber valgte denne dog helst Saaoanne, som ved sit Kjendskab til Loven og ved et stille , fra Verden afsondret , beskuende Liv var mest tilgjengelige for Hans Rost . Desuden holdt Kongerne og de Store blandt Folket Lcerere til sine Born . Kong David lader sin Son og Arving Salomo undervise af Profeten Nathan . Men i de fleste Familier paahvilede det Forceldrene at undervise sine Born i Loven . Den sknlde de indskjcerpe sine Born , for at de aldrig siulde glemme den . Saaledes var det endnu i Herrens og Hans Apostles Dage . Hellerikke da gaves der egne Skoler for Born . Gcnnaliel holdt vel en Skole i Jerusalem ; Paulus havde siddet ved hans Fodder . Men dette var dog kun et lidet Antal af unge Mcend . Og om <b>Pigernes</b> Undervisning vide vi slet Intet . De Synagoger eller Skoler , som saa ofte omtales i Evangelierne , vare ikke for Born , men for Vorne . Da loderne kun havde det ene Tempel , blev Loven forelcest dem i disse Skoler , der fandtes i de forskjellige Byer . Bornenes Undervisning , forfaavidtsom de fik nogen , hvilede , som sagt , ncesten udelukkende i Foreldrenes Hcender . Blandt Hedningerne var Underviisniugsvcesenet i en endnu misligere Forfatning . Hos Grcekerne og Romerne besorgedes Bornenes Underviisning i rige Familier af en Trcel . Han var kjobt , og naar man ikke lamger havde Brug for ham , blev han atter folgt . I de hoiere hedenske <b>Kundskaber</b> underviste Filosoferne ( de Verdsligvise). De samlede ofte store Skarer af Disciple omkring sig og havde sine egne Skoler . I Apostlenes Gjerninger omtales en <b>vis</b> Tyrannus ' s Skole , i hvilken Paulus prcedikeoe under sit Ophold i Byen Efesus . Den christne Kirke er det forste store Samfund , som fra gammel Tid har cm- Mt det for sin Pligt at undervise samtlige sine Born om

<h4>365</h4>Drenge og <b>Piger</b> deriblandt , som intet Hjerte have for sine Lcerere , som ikke lamger barnligt hengive sig til dem , men i det Hoieste indtage en klog og beregnet Stilling ligeoverfor dem . I deres Ncervcerelse <b>vise</b> de dem vel en nsdtorftig Lydighed ; de have vel endnu nogen Frygt i sig for Lovens Bogstav ; men der er ingen hjertelig Lydighed . Den barnlige VErefrygt , som Bsrn skylde sin Lcerer , fattes hos altfor Mange . Af Mrefrygten ogldmygheden bor der opstaae en übetinget Lydighed hos Barnet . Lammene folge Hyrden . Ogsaa her gjcelder , hvad strevet staaer : „ I Born , adlyder Eders Forceldre i Herren ! thi dette er ret . " Lcereren ndover nemlig en Deel af Forældrenes Embede . Og for at lcegge alle Born dette ret kraftigt paa Hjerte , siges der i Brevet til Ebrceerne : „ Adlyder Eders Veiledere , og vcerer dem lydige thi de vaage over Eders Sjele som de , der skulle gjore Regnstab at de kunne gjore det med Glcede og ikke sukkende ! thi dette er Eder ikke gavnligt . " Enhver Ulydighed mod Lcereren bor Forældrene betragte som begaaet mod dem selv . En Folge af at der hos Bornene nuomstunder ofte er Mangel paa hine to Grundbetingelfer er , at det ogsaa mangler dem paa de forste barnlige Dyder i Forholdet til Lcereren , nemlig Kjcerlighed og Taknemmelighed . Intet Menneske kan stjcenke os noget Stsrre end den stjcenker os , som gjor os fortrolige med Herren , som med Kjcerlighed , Alvor og Bon veileder os paa Frelsens Vei , som fortscrtter vor <b>Uddannelse</b> til Lighed med vor Herre lesns Christns , som forer os ncermere hen til Ham , der er Hovedet , nemlig Christns . Isandhed , et saadcmt Arbeide , et scmdcmt Hjerteog Sjelearbeide er al Tak vcerdt . Med en Lcerer er mcm ikke fcerdig , naar man har betalt ham Skolepengene . Man kan ikke betale en Lcerer som en Hacmdvcerker . For Hjertearbeide tilkommer ham en Hjertetak . Paa en rorende Maade have Hedningerne ofte viist sig taknemmelige mod sine Lcerere. Den romerske Keiser Markus Aurelius overoste ei alene sine Lcerere med Rigdomme og Vcerdigheder : efter deres Dod fatte han , Hedningen , endog deres Billeder blandt stne Huusguder . Dette var just ikke saa urigtigt gjort af ham , som en Hedning ; thi i Sammenligning med de andre Guder skyldte han ialfald sine Lcerere langt Mere end disse . Den gamle romerske Viismand Seneka ' s Disciple bragte ham en Dag Gaver . Nu var der En iblandt dem , som var saa fattig, at han ikke kunde bringe ham Noget . Denne traadte da grcedende frem for sin Lcerer og sagde : , Jeg eier Intet ; men zeg brmger tng mig selv , mit Hjerte og min Kjcerlighed . " Og nn christne Born , christne Disciple ! De have jo dog ialfald annammet Mere af gudfrygtige Lcerere end hine Hedninger af

<h4>645</h4>Tag . Te vide endnu ikke , med hvilken Skade for sin <b>Sjel</b> og for et heelt langt Liv de ofte maae betale en faadan Frihed . De forstaas endnu ikke , at de ved at arbeide iet enkelt Fag ingen <b>Uddannelse</b> faae for hele Livet . Det hnuslige Liv maa nodvendigviis gjore en <b>Kvinde</b> mere stikket sor hendes tilkommende Huns end Livet i en Fabrik . Hvad kan en <b>Kvinde</b> forrette i sit Huns , der som <b>Pige</b> har forfærdiget Pynt , Blomster , Krsller , Cigarer , der som <b>Pige</b> har staaet ved en Hurtigpresse og paalagt og aftaget Arkene eller tilieet en Spindemaskine eller gjennem hele Maaneder drevet et eensidigt og eensformigt Arbeide i et Sukkerraffinaderi ? I mene , at saadant Arbeide lonner sig bedre ; men det er ikke Tilfaldet . I gaae glip af den stille huuslige Gloede og ar den paa lcenge tilvant Arbeids grnndede <b>Uddannelse</b> til en brav og dygtig HuuZmoder ; at , I tabe osts endnu langt Mere ! " Naar Eders eget Huus siden falder Eder for nille , naar I hverken have Sands sor eller Ovelse i dets mangebaande smaae Sysler , naar I ikke ere Bestyrelsen ar en simpel snmsboloning vorne , naar den cegtestabelige Fred strander ^ paa ' disse Skjcer , da ville I engang indsee , yvor nor Skade I have hast af den tidligs Selvstændighed og storre Fortjeneste . Fordi Josef havde tjent , kunde han uden herske ; sordi han havde lcert at sorvalts en Andens Gods , kunde han siden forvalte sit eger . Men det Kosteligste , han bragte med ng fra sin Tjeneste hos Votisar og i Fcengslet , var Ydmngheden . Og lcegger nu rigtigt Mcerke til vort Brev ! Onesimus har vceret Trcel hos Filemon , en Chri , ten i Kolossce i Anen . Han har lobet bort ira ham . Hvorledes han maaskes har flakket om , hvor meget Ondt han maasiee bar doiet . derom siger Skriften ikke et Ord . Om det har gaaet ham vel eller ilds , derom staaer der Intet skrevet . Men rimeligviis har han lidt Ondt . En saadan bortwmt Trcel betragtedes jo af Romerne som et vildt Dyr , der var sluppet ud af sit Buur . Icegerne med alle nne vare i Hcelens paa ham . Onesimus flygter til Rom . I den store Vrimmel as to Millioner Mennesker kan en flygtning lettest skjule ng . Vistnok gaaer ogsaa der hans <b>Sjel</b> lettest tabt . om ellers Herren endnu har nogen Deel i den . Og Herren havde endnu Deel i Onenmus s SM ; ia , Han vilde have den heel og holden . Han lod ham paa nn vildfarende Vei komme til den fcengslede Paulus . Og Paulus briuger ham ved sin vceldige Prcediken , vceldig ? elv l bans Lcenker , til den Herre lesum , der alene formaaer at gjsre Mennestet fri . Kun den , fom Ssnnen faaer frigzort , er virkeligen fri . Onenmus erkjendte , at den Frihed , han svigagtigt bavde forstaffet sig , var en falsk Frihed . Han lod

<h4>672</h4>saa sess at erholde en gudfrygtig Mester eller Herre . Den Luft , du hele Dagen bevceger dig i , er af stor Indflydelse paa din Helbred . Hvorsomhelst du kommer hen , saa see til at finde dig et Par gudfrygtige Omgangsvenner , med hvilke du kan samtale om Guds Ord og din Frelse . Kvor langt du end kommer bort fra dit Foedrenehjem , saa flriv flittigt hjem . Ethvert Brev er en Samtale med dine Forcrldre . I ethvert Brev maa du paa en <b>vis</b> Maade troeds frem for dem . Derfor maa du ogfaa gaae paa Naad med dig selv om hvorledes du skal troede frem for dem . Men for Alt , hvad du gjer , vogt dig for al Lsgn i dine Breve ! Et Lsgnbrev bringer dig mindst hundred Mile lamger bort fra dine Foroeldre , uden at du behsver at gaae ud af Byen . Ja skriv ret flittigt hjem . Glem ikke det fjerde Bud , fordi om du er paa et fremmed Sted . Din Frelser tog det med sig lige op paa Korset ; tag derfor ogfaa du det med dig til det sidste Sted , du soetter din Fod paa . Gjer dig ingen Bekymring for den dermed forbundne Udqift ; alle Forceldre betale gjerne nogle Skillinger for et Brev fra sin SM . Har du imidlertid en syg Fader eller en gammel <b>Moder</b> , wm har vanskeligt for at tilveiebringe Brevpengene , faa betal dem felv . Sult heller for en Gangs Skyld . Naar Postbudet bringer din <b>Moder</b> Brevet , seer hun allerede af Frimcerket , at du endnu er en tro og kjcerlig Sen . Arbeid derhos ufortrødent . Flyv ikke fra den ene By eller Mester til den anden . Der groer ikke Groes paa den Steen , som ofte vceltes . En Svend , fom opslider sin Fortjeneste i Stevlesaaler, loerer Intet . Vcer Ydmyg ; thi kun Ydmygheden lcerer Noget . Hoffcerdigheden kan Alt og dog tilsidst Tcenk endelig stedse paa at det egentlige <b>Maal</b> for din Vandring er den salige Evighed , og hvorledes du stal komme d : d , det loerer Ingen dig faa sikkert som dit Nytestament . Ligesom vi , mine elskede Loefere have vceret med den unge Mand paa hans Reiser og Vandrinqer , saaledes ville vi ogsaa vcere tilstede , naar han efter flere Aars Forlob kommer tilbage til sine Foroeldres Huus . Det er en vigtig Dag . Hvor opmærksomt betragte ikke hans Fader og <b>Moder</b> ham ! Hvor utaalmodigt vente de ikke paa at han rigtigt stal oplade sit Hjerte for dem ! De ville vide , hvad han er bleven til . Og hvad erfare de da ? Der er fire Slags faringer , de isaafald kunne gisre . Een Deel af de unge Moend er vandret ud fra Hjemmet uden at taae Noget med ug . Verden og dens Lyst eiede deres Hjerte " , da de droq dort , og de e : e det fremdeles , naar de komme tilbage . Vierts er kun blevet csldre , Haarders og fortroligere med hynden . Da staae Froenderne sorgmodige hos den Hjemkomme.

<h4>734</h4>flet ikke tage imod nogen Betaling derfor . Han sagde til Mraham : „ Hvad er det mellem mig og dig ? Det er jo en reen Übetydelighed i <b>Formaal</b> til vore Formuer . " Der var saaledes ingen Tvivl om at en af disse Kana ' aniter- W ? ers Dstre vilde faa et godt Udstyr . Men desuagtet stal Elieser ikke tage nogen <b>Hustru</b> til Isak blandt deres Dotre . Hvorfor ikke ? Fordi disse vare hedenske , afgudiste og csgte Verdensbarn . Abraham havde vistnok de bedste manker om Isak , han havde ogsaa den Tillid til ham , at yan vilde forblive den levende Gud tro . Men han vilde alligevel ikke bringe ham nogen Snare i Huset . Han skulde ilts tags nogen <b>Hustru</b> , som han forst maatte omvende fra de dods Afguder til deu levende Gno . Mgtessabet er ingen Opdragelses- eller Forbedringsanstalt , men tvende troende Hierters Pagt i Herren . Og hvem kan svare for at ikke en hedensk Hnstru og vidtlsftige hedenste Svogerstabsforbindelser tnnde have boiet Isaks Hjerte til Afguderne , faaledes fom det gik Kong Salomo , Davids Son ? Abraham vilde holde Nt Huns reent , og det gjords han ret i . Naar du derfor stal trMs dit Valg , saa tag hellerikke du dig nogen <b>Hustru</b> af Kana ' aniternes Dstre , tag dig ingen Mand af Kana amternes Eonner . Dette er det forste Naad . Christel'at talt vil det , igs det Samme som : tag dig ingen vantro Mgtefcelle . Har Herren af Naade bragt dig til Troen , saa styrt dig ikke forscetlig i Fare . Toenk ikke : ' „ Jeg stal nok omvende min <b>Hustru</b> eller min Mand til Herren . " Anse dlg ikke for klogere end Abraham og Paulus . Deu Sidste stger : „ Hvad veed du , Mand , om du kan frelfs Hustruen ? Hvad veed du , <b>Hustru</b> , om du kan frelse Manden ? Hvo wm vandrer i Ydmyghed for Herren , leger ikke med sin Melse . Han veed , hvilken Indflydelse Manden har vaa sin <b>Kone</b> og <b>Konen</b> vaa sin Mand . Han veed , at den vantro <b>Aand</b> , som boer i en Familie , har en frygtelig Magt til at dcempe Livet hos et nyt Medlem af famme . Vogt dig desaarfag for denne Tids Afgudsdyrkere . Et Huus , hvis Guder ere Penge Kun , t , fiin Verdensdannelse eller ogfaa raaKjsdsly,t, af det bor ingen christelig Mand tage sig en <b>Hustru</b> ingen christelig <b>Kvinde</b> en Mand , ialfald ikke , medmindre di sienfynkgen have afstaaet fra Husets Synder og bekiendt ng til Herren . < s ikke undlade at tilfoie en Advarsel mod b andede Mgtestaber . Saare sjelden have Mgtestaber mellem til noget velsignet , begge Parterv frelse fremmende Huusliv . I Fsrstningen men r det unge Par at Intet stal kunne virke forstyrrende paa derev cegtestabelige Lykke . Men Forskjellen gjor sig allerede

<h4>738</h4>Groendserne ; men disse gjore sig dog senere gjcrldende , og Misforholdet virker da forstyrrende gjennem hele Livet . Der bor heller ikke ved din Forlovelse opstaae nogen Kloft mellem dig , dine Forceldre og Sodstende . Jeg maa derhos lcegge dig paa Hjertet , at man for Forlovelsen bor have lcert hinanden grundigt at kjende . Man pleier at sige , at man ikke bor flutte Venskab med Nogen , forend man har spiist een eller to Skjcepper Salt med ham . Et Wgteflab er noget langt Mere end et Venskab . Fra en Ven kan jeg trcekke mig tilbage , men derimod ikke fra en <b>Hustru</b> eller Mand . Vogt dig desaarsag for en overilet Forlovelse . Jeg har spurgt mange Mgtepar , hvis Fred var bortvegen og fom flet ikke kunne stemme overeens , hvor lcenge de havde kjendt hinanden , for de blev forlovede , og da har jeg faaet til Svar : „ Fire Uger , sex Uger , otte Uger . " Og tilfidst kunde de endog sige : „ Vi kjendte hinanden i Grunden flet ikke . " Spurgte jeg videre : „ Talte I nogensinde med hinanden om Eders Troes Grundvold og Haab , bad I nogensinde med hinanden , gik I nogensinde til den hellige Nadverd sammen ? " saa lod Svaret„Nei" og atter „ Nei " og endnu engang „ Nei " . Hvor stal der da komme nogen Enighed fra ? Forlovelsen , der sluttes : Kjodet , af Kjsdets Lyst eller verdslig Beregning , deler alt Kjods Lod . Alt Kjod er Groes , og al dets Yndest som Markens Blomst . Unge <b>Piger</b> , som ingen Lyst bave til det wlle huuslige Liv , som elste Balsalen hoisre end Stuen og Kirken , som med Diana gaae ud for at see paa Hedningernes Dotre , blive aldrig gode Hustruer . Din Bruds kjcereste Steder bor vcere Herreus Huus og hendes eget s , uus . Kun den <b>Pige</b> bor vcere din Brud , som kan sige til dig som Ruth : „ Hvor du gaaer hen , vi ! jeg gaae , og hvor du forbliver , vil M forblive : dit Folk er mit Folk , og din Gud er min Gud . " Kun med hende kan du vcere <b>vis</b> paa at komme til at leve i Fred og Enighed . At imidlertid en saadcm Pagt aldrig bor sluttes uden Forceldrenes Samtykke , at disfe endog bor adsporges for Bruden selv , og at man forst maa have sikret ug deres Velsignelse , er Noget , hvorom ingen Christen kan vcere : Tvivl . Det Bifald , som efter Forlovelsens Indgaaelse afnodes dem , er intet Bifald , den Velsignelse , som da gives , mgen Velsignelse . Du bor agte deres Ord saa hoit ctt du ogsaa stille ydmyger dig under deres Nei . Og vil du vye dem denne Mre , saa bor du allerede adsporge dem , naar den forste T : lbsielighed fremspirer i dit Hjerte , og forinden du endnu har gjort nogetfomhelst utvetydigt Skridt henimod <b>Maalet</b> . Til disse forste og vigtigste Fordringer tomme endnu nogle andre . Mgtefolk bor ogfaa med Hensyn

<h4>961</h4>for stne Slægtninge og Bekjendts . Naar Herren «åbenbarer sin Herlighed paa dem , naar Han gjsr dem salige midt i deres Elendighed , naar Han giver dem Hovedpryd istedetfor Aske , Gloedes Olie istedetfor Sorg , Lovpriisnings Kledebon istedetfor en vansmægtende <b>Aand</b> , naar de staae der som Retfærdighedens Terebinther , Herrens Plantning , til Hans Forherligelse , naar den Halte springer som Hjorten, og den Stummes Tunge jubler , o , hvor prediker da ikke denne den Hellig-Aands Naadevirkning ! o , hvor blive da ikke Herrens Gjerninger aabenbarede ! De kunne imidlertid ogsaa predike og vidne paa andre Maader i sine Familier. " De kunne ststte Huset med sine Bønner og frembcere dets Ve og Vel for Herren . Ved Hjelp af egen dyb Erfaring kunne de <b>vise</b> Ssstende , Broder og Sosterbsrn saavelsom Andre Salighedsveien . De kunne forkynde fin hele Frcends- og Vennekreds Guds underfulde Gjerninger . Her i Byen ( Leipzig ) levede for ikke lenge siden en gammel blind <b>Kone</b> , fom i Vantroens Msrke blev en Vederkvegelse for mange tsrstige Sjele . Hun kunde vidne ganske anderledes om det Lys , som har skinnet i Msrket , end de fleste Mennester med klare Vine . I Aaret 1739 sendte Brsdremenigheden to Missionerer til Vestindien . Den Ene , Albin Theodor Feders havde studeret ved tre Universiteter og var en dygtig Theolog , og man gjorde sig store Forventninger om ham . Den Anden , Christian Gottlieb Israels , var en halt og pukkelrygget Skrceddersvend . Den stakkels Karl havde et ynkeligt Udvortes ; men med hans Indre stod det herligt til . Der var han retvoxen som en Gran , strakte sig i fast Tro og barnlig Kjcerlighed opad til sin Herre og havde intet andet Vnste end at tjene Ham blandt de arme Negre . At han selv formaaede Noget , var han langt fra at indbilde sig ; han kjendte sin egen Skrøbelighed ; men Herren skulde vcere sterk i ham . <b>Maalet</b> for deres Reise var Ven Thomas . Paa et dansk Fartsi reiste de til St . Eustatius , hvor de gik ombord paa et engelsk Barkskib , som skulde bringe dem til deres Bestemmelsessted . De led imidlertid Skibbrud paa Klippesen Skrop . Størstedelen af Mandskabet reddede sig i Storbaaden og kom iland paa Ven , hvorimod de to Missionerer maatte blive tilbags paa Skibet , Fsrend dette gik under , havde de fundet et midlertidigt Tilflugtssted paa et lidet Skjcer . Feders forsogte at naae Land paa en Rekke Stene , som rageds op over Vandet, men gled og fandt sin Dod i Bolgerne . Den halte Israels derimod blev omsider trnkken iland paa Ven ved s , jcelp af et udkastet Toug og faaledes reddet . Da tenkte Mange : „ Den dygtige Arbeider ligger paa Havsens Bund

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013010208009'target=_>Müller, Heinrich, 1860, Himmelsk Kjærlighedskys</a><h4>1461</h4>Saa vidt man af Skriften kan mcerke , holder Gud den folelige Ssdhed tilbage . 1 ) Naar Menneskets Skrsbelighed er saa stor , at den ikke kan taale den ; thi man har fundet Mange , som have opgivet Aanden over , at de have havt hoie og forunderlige Guds Virkninger i.deres Sjoele , mere end deres menneskelige Skrsbelighed kunde boere . Men naar den fromme og <b>vise</b> Gud seer , at saadanne Mennesker , ved at nyde hans Ssdhed , overstride <b>Maalet</b> , og endog drikke sig drukne deraf , saa gjsr han , ligesom en forsigtig Fader . Naar denne har megen kostelig Viin i sit Huus og begiver sig til Hvile , saa tsge Bsrncne for sig , drikke begjoerlig og blive drukne ; men naar Faderen staaer op og erfarer , hvad Bsrnene have gjort , saa slaaer han dem med skarpe Riis , saaat de siden fole ligesaa store Smerter af Riset , som de tilforn havde Gloede af den gode Viin . 2 ) Naar vi ere komne saa vidt i Christendommen , ikke mere soge Verden , eller ere bange for Arbeide og Troengsel ; thi Gud holder den samme Huusholdning t sit Rige med de Fromme , som en <b>Moder</b> med sine Bsrn . Saaloenge et Barn endnu er ungt og spcedt , saa opfsder Moderen det med Melk og Hvedebrsd ; men naar det bliver stsrre , saa afvcenner hun det , ikke fordi hun vil fordoerve det , men forar lcere det at spise Haarders Mad , saa at det kan vore , og blive stort . Saaledes giver Gud de Nyomvendte lutter Sukker , Honning og Sodme , lokker dem dermed til sig , fordi de endnu ere svage , gjor dem derved kjede af den sode Verden , og den bittre Christendom sod for dem ; han lader sin guddommelige Kjoerligheds Lue broende i dem , drager og bsier deres Villie med allehaande sode Velgjerninger , at de stulle erkjende , tove og prise hans naadige Villie imod dem . Men de gamle Christne giver han

<h4>1664</h4>som David beder i den 51 de Psalme : Gud , skab i mig et reent Hjerte og fornye en stadig <b>Aand</b> i mit Inderstes ) . Hvad en Mester har gjort med sine Hcender , det behager ham . Det Menneske er Christi <b>Dannelse</b> , som Paulus stuver : Indtil Christus faaer sin <b>Dannelse</b> i Eder 2 ) . Nu er Christus Grunden til alt Guds Velbehag : Denne er min elskelige Ssn , i hvilken jeg har raaber Faderen selv fra Himmelen . Det nye Menneske er Guds Billede , som Paulus skriver om det ; Det er stabt efter Gud i Sandhedens Retfærdighed og Hellighed ^ ) . Ligheden er en <b>Moder</b> til Kjcerlighed , Kærligheden til Velbehag . Joseph behagede sin Fader i den brogede Kjortel , som han selv havde givet ham . Gud beklcedcr os med Christo ; thi saa mange som ere dsbte paa Christum , de have ifsrt sig Christum ; naar vi da ere ifsrte denne Prydelse , saa forlyster Gud sig i os ; hans eget Barn , hans egen Prydelse maa jo behage ham . Hos Propheten Esaias finde vi disse trsstelige Ord : Ikke lcrngere stal du kaldes : den Forstudie , og dit Land ikke loengere : det Dde . Nei , du stal hedde : den af mig Elstte ; og dit Land : Mgtehustruen ; thi Herren vil elste dig , og dit Land stal forenes med sin Mgtefoelle . Thi som Inglingen cegter en Ms , saa vil din Opbygger cegte dig ; og som Brudgommen gloeder sig ved sin Brud , saa stal din Gud gloede sig ved dig ^ ) . Jeg tor sige , at jeg ei i den ganske Skrift har lcest dejligere Ord om Gud , end disse , i hvilke han er os afmalet som en kjcerlig Ven og Brudgom , der ikke veed at beholde sin Kjcerlighed , men udsser den over sin Brud i Hobeviis i hendes Skjod , kysser og kjcertegner hende , ja veed neppe for indvortes Lyst og Glcede , hvorledes han stal tee sig . Forestil dig en saadan Ven og Brudgom , saa har du et Billede paa , hvorledes Gud tilbyder sig selv til en troende Sjcel , hvilket dog kun er et Billede og en Skygge , altfor ringe til fuldkommen at kunne afmale Guds overvættes rige Kjoerlighed . Ak ! hvor kjcerlig omdsber han sit Barn , saaar den Forladte faaer en Mgtchustru , og Orkenen bliver til en Lysthave .

<h4>2385</h4>tcenker : naar stal jeg staae op ? naar stal Natten naae sit <b>Maal</b> ? og jeg bliver mcet af at kaste mig til Morgengrys Og David : Du holdt mine Dine vaagne , jeg var forfcerdet , og talede Jeg vaager og er som den eenlige Fugl paa Taget ^ ) . Ak ! hvor mangen Sjcel maa ikke som en forladt Fugl kurre , jamre , sukke og groede den hele Nat igjennem . Denne indvortes Anfoegtelse og Angest har Guds Kjoerlighed af Evighed besluttet over de Fromme , og lader den ogsaa i sin Tid komme , ja Angest over Synden og <b>aandelig</b> Bedrøvelse virker han selv ved den Helligaand i os ; thi at vi ere bedrsvede over Synden , det kommer ikke af Kjsd og Blod , men af den Helligaand , naar Denne afsiger Dommen over Synden i Hjertet baade af Guds Lov og ChristiKors ; derfor kalder Paulus denne Bcdrsvelse en Bedrovelse efter saave ! fordi den er af Gud , som ogsaa fordi den leder til Gud , ligesom Vandrenderne , der baade gaae fra Bremden og <b>vise</b> Veien til Bronden . David kalder den Guds Vrede og Harmes ) . Bar Bedrøvelsen ikke af Gud , hvor kunde han da have frimodig taget sin Tilflugt til Gud . De Hellige gjore , som der staaer hos Propheten Hoseas : Kommer , og lader os vende tilbage til Herren ; thi han har ssnderrevet , og stal lcege os , har siaaet , og stal forbinde os « ) . Naar Lcegen slaaer den Syge et Saar , der stal tjene ham til Sundhed , da veed han ogsaa bedst at lcege det , og den Syge kan sikkert betroe sig til ham . Gud , der saarer Sjcelen , veed bedst , hvad for et Lcrgeplaster han tilskikker den , han veed , hvilken Trsst der stal fslge paa Lidelsen . I den 116 de Psalme kaldes Anfægtelserne en Angest fra Gud , ligesom en <b>Moder</b> ofte forfoerder sit Barn med Larm , men det er kun en Kjoerlighedsleg . Ezechias og Job tilskrive ogsaa Gud det . Hiin siger : Han , nemlig Herren , suger mig tor ud , denne kalder det Guds Pile . Ligesom Legemets Sygdom kommer fra Gud , saaledes ogsaa Sjcelens . Legemet har sine Sygdomme , Smerter , sin Hunger , Tsrst , Frost , Ns-

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2007061902001'target=_>Munch, P.A., 1849, Illustreret Verdenshistorie</a><h4>5210</h4>Det var ikke uden stcerr Modstand , at Gregor VII formanede at gjennemfore denne Lov . Mangfoldige Prester modsatte sig aabenbart , og da f . Er . Erkebiskop Sigfred af Mainz forkyndte de pavelige Dekreter , gjorde Presterne en saadan Opstand , at Erkebispen kom i Livsfare . Paa en Kirkeforsamling i Paris blev det endog formelig besluttet , at denne Lov ikke skulde efterleves, og i Burg o s i Spanien blev en pavelig Legat mishandlet , som vilde fratage Presterne deres Koner . Dog de Gejstliges Opposition var forgjeves. Gregor havde et Middel tll atgjennem- - fore sin Beslutning , som var virksom nok . Medens han rolig lod Stormen udrase , ssrgede han overalt for at underrette Lcegfolk om den Banlysning, som truede Enhver , der horte Messe hos en ugift Prest . Derved bragte han Folket , som allerede var indtaget for hine Forestillinger om den ugifte Stands Hellighed , og som i hine Prester alene vilde see Hellige , aldeles Paa sin Side . Paa mange Steder rejste derfor den simple Mand sig mod de Gejstlige og tvang dem til at frasige sig sine Koner . De som undssoge sig herfor , bleve forhaanede , pryglede , udplyndrede , drevne i Landflygtighed, ja paa enkelte Steder endog martrede til Dode . Saaledes bekvemmede da lidt efter lidt den Ene efter den Anden sig til at forssyde <b>Kone</b> og Born , og Greg ors <b>Maal</b> blev i Tidens Lob virkelig naaet . Kun i de nordiske Riger formanede Colibatet ikke ganske at trcenge igzennem , da den pavelige Autoritet , formedelst Afstanden , ej kunde <b>vise</b> sig faa virksom som i Lande , der laa ncermere Rom .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014011608145'target=_>James, John Angell, 1868, unge Qvinde</a><h4>60</h4>Familielivets , og derved hele Samfnndslivets , snude Udvikling og sande Velfcerd har sine forste og dybeste Rsdder i Moderhjemmet . Det er fra Moderen , at Barnet faar sine forste , fleste og varigste Indtryk og Grundlaget til sin Udvikling og <b>Dannelse</b> for hele Livet , ja , for en stor Del for Evigheden ; dct er i Hjemmet , i Barndom , mens Morgen , at Barnets Charakter egentlig dannes , og Intet tan til dette Oiemcd veere vigtigere , end at <b>Qvindcn</b> , som har den store Bestemmelse at give de Slcegter , som sodes , den fsrste stjcrrmcnde Pleie og at fvrede den sande Kjcrrligheds Sollys og gyde den sande Fromheds Morgendug over Familielivets Hjerteblade , at hun dertil bliver forberedt ved Herrens <b>Aand</b> ; og i denne Henseende er en god , en udmarker Veiledning af en hsit oplyst og erfaren Lcerer af megen Vcrgt ,

<h4>62</h4>Fremdeles er det for Familielivet , saavelsom for Taiufunds . livet , übeskrivelig fljsnt og velgjsreude , naar Oviuden ved en rigtig Ledelse er dannet og forberedt til at opfylde sit vigtige Kald som <b>Hustru</b> for at kunne vcere Hjemmets storste Prydelse og den virksomme, milde <b>Aand</b> , fra hvilten det huslige Liv henter Trivsel og Lykke nnder den sande Kjcerligheds , den oplyste Forstands og den oprigtige Bsns Magt , som Apostelen siger : „ Hjertets skjulte Menneske i en sagtmodig og stille Aands uforkrænkelige Vcesen , hvilket er meget kosteligt for Gud . " 1 Pet . 3 , 1 - 5 , Det er ogsaa ved cn saaledes beskaffen <b>Dannelse</b> og SMsretning , at den nngcQvindc , saavcl som Mennesket i Almindelighed , ikke blot for sin egen Del bliver allerlykkeligst baade i Fcedrehjcnnucl og under Udsvelscn af de forskjellige Kald i det almindelige Liv , men ogsaa faar Sind og Evne til at medvirke til Andres Vclfcerd og til at have cn velgjsrende Indflydelse paa Mange . I alle Henseender giver denne Bog fortrceffelig Undervisning og Netledning , At cn saadan tan tiltrcenges, var let at vifc ved Paavegclsc af mange forgelige , stjeeve Retninger , der crc meget sædvanlige , iscrr i Henseende til den avindelige Ungdoms Opdragelse , eller Mangel paa Opdragelse , og den urigtige Behandling , dcn faar , forudcu mange Afvcie , fom erc at beklage .

<h4>107</h4>Til Hedenskabets og Muhamedanismens undertrykkende og grumme Behandling af QvindcWnnct fremstiller Christendommen en st ' jM og elskelig Modsætning , idet den til Jødedommens delvise Bestemmelser angaaende Qvindeus Rettigheder tilfKcr en fuldkonnnen Erkjendelse af hendes Fordringer . Det er vor hellige Religions Wre og et Bevis paa , at den er ndfprungen af den guddommelige Kjcorligbed, at den er en Fiende af al Undertrykkelse , at den giver Enhver , hvad ham tilkommer . Den trader Ingens Rettigheder nnder Fadder , den modstaar alt Ondt : men Ingen staar i stirre Forpligtelse til den end <b>Qvinden</b> for dens milde og retfardigc Styrelse . Ved Christendommen har <b>Qvinden</b> erholdt sin moralske og sociale Indflydelse , ja ncesten hele sin Tilværelse som et Samfundets Medlem . Qvindens Sjal , hvilten Mange af Fortidens Philofopher , Lovgivere og Bise anfaa for nt vare af ringere Vard og ude af Stand til at modtage Dannclfe , er dog bleven udviklet under Christendommens Beskyttelse . Christi Evangelium , dette glade Befrielsens Budskab , er nedsteget i denne forsømte Grnbe , hvilken endog de <b>Vise</b> have soragtct at oparbeidc , og har derfra bentet op mangen uskattcrlig Juvel , straalendc af Snillets Ild og af den cbristelige Kjarligheds elskelige Farveglando . Cbristendonnnen bar gjengivet <b>Qvinden</b> hendes tabte eller bortrovedc Rettigheder og indsat de klareste Wdelstene i bendeo Haderokrone . Hendes tidligere Forfald forklarer idetinindste for en Del Übestandigheden af tidligere Tiders <b>Dannelse</b> . Det er nmnligr for Samfundet vedvarende at bolde sig oppe , naar dets <b>Qvinder</b> crc sornedrcde til Slaver . Hvorsombelst den <b>qvindelige</b> Slagt ansees sor underordnet , der baner Samfundet Dplooninaens Spire i sit S tjod og er berovet til alt virkeligt Fremskridt . Det er nmuligt , at Indretninger og Sædvaner , bvilke bindre eller qvale ' ) ! atnreno Instinkter og kranke Love , knnne trones med endelig Fremgang . Samfundet niaa forandre sit vdre lldfeende , inaa freuistillc Rigdommens Glands , Smagens Forsinelse , Vvnnsteno Fo , stionnelse eller Videnskabens og mere vardifnlde Frernftridt ; nien hvis Qvindcns Sjal forbliver nndviklet , hendes Smag

<h4>124</h4>bM hun ikke fryde sig over det Ord , ' „ Hvo , son , stiller sig fra sin <b>Hustru</b> uden for Hors Sag , gjor , at hun bedriver Hor , og hvo , som tager en Fraskilt tilcrgte , bedriver Hor . " Ildernes Mie Frihed i deune Henseende blev saaledes ikke billiget af Christendommen ; Drre Sikkerhed gaves Qvindens Wre og Lykke , og en dybere Grund lagdes for den huslige Lyksalighed . Om det itkc var for noget Andet end dette , fortjente Chriftendommcn Menncsteslcegtens Taknemmelighed. Men det er blot en Del af dens Wre , at den afskaffede Brugen af mange Hustruer ; endun mere er det , ar den beskytter denne ene Hustrues Rettigheder , Anseelse og Lykke . Den beskytter hendes Hjem for utitbo ' rlig Indtrængen og sikkrer hende selv en tryg og urokkelig Besiddelse deraf . Jeg vil ogsaa omtale den lige Andel i Religionens Velsignelser , som skjenkes den christne <b>Qvinde</b> . Hvor tydelig og bestemt har ikke Apostelen tilsitkrct ogsaa <b>Qvinden</b> Del i alle Christi Velgjerninger , naar han siger : „ Her er ikke IM eller Grcekcr ; her er ikke Trcel eller Fri ; her er ikke Mand eller <b>Qvinde</b> ; thi I ere Alle En i Christo Jesu . " Dette er et Fribrev , som skjenker <b>Qvinden</b> alle Frelsens Velsignelser; dette er Beviset for Qvindcns leynlighed med Manden for Gud ; dette er Qvindens Rettigbed til sin sande Plads i Samfundet . Der findes ingen Velsignelse , nødvendig for at vinde det evige Liv , hvilken ei i famme <b>Maal</b> tildeles <b>Qvinden</b> fom Manden . Det er en almindelig Tro blandt Muhamedanerne , at <b>Qvinder</b> itkc faa Adgang til Paradiset ; ere istedet scerlig skabte for denne Plads . Hvor fornedrende er ei en faadan Tanke ! Men den er i fuldkommen Overensstemmelse med Muhamedanismens <b>Aand</b> , der betragter <b>Qvinden</b> mere som en Slave for Mandens lidenskaber end som hans Ledsagerinde gjennem Livet . Christendommen stiller <b>Qvinden</b> ved Mandens Side , som <b>Hustru</b> , som Datter , som Tjenestepige ; ved Familiearnen, i Templet , i Menigheden . Mand og <b>Qvinde</b> mMs ved Korsets Fod og skulle ogsaa mMs i de Saliges Voliger.

<h4>131</h4>den mindste af de Udmcerkelscr , som vare bestemte sov dem as vor bellige Religion , De indloge en fremragende Plads i de Arbeider og Svvler , som paataldtes af Nodvendigbeden i de apostoliske Tider sor Udbredelsen as Ebristi Rige paa Jorden , Hvor lcrrerig og rorende er der ikke at bore Pauli Opfordrings „ Antag dig disse <b>Qvinder</b> , tbi de bave stridt med mig i Evangeliet " . Hvilken Merngde af Navne paa udmeerkedc <b>Qvinder</b> og deres Arbeider sinde vi ilte i Romerbrevets 10 dc C.ipitel . „ Priscilla , mm Medarbeider " ; „ Maria , som bar arbeidet meget sor os " ; „ Trvpbena og Trvpbosa , som bave arbeidet i Herren " ; „ Pboebe , som er Menigbedens <b>Tjenerinde</b> i Kenchrece " , der sendtes til M.nigbeden i Rom og betroedes en saa vigtig Sendclse , som at overbringe den det af Apostelens Skrifter , bvilket , om vi bave til at sammenligne den ene Tel as Skriften med den anden , er den mest dvrebare af den guddommelige Aabenbaring ! I Forbindelse med alt dette er det ndensor al Tvivl , at <b>Qvinderne</b> i den sorste Menigbcds Ttdcr ikke alene begavedes med Aandens underbare Krast at propbeterc , men ogsaa anvendtes som Diakonisser . Ryere Tiders ebristnc Kirker cre ikke vedblevne med at cere den <b>qvindelige</b> ( ibarakter . Ebristendommens Barndom stod i denne Henseende roran vor Tid ved at anvende <b>Qvindcn</b> i Kirkens og Menigbedens Tjeneste og i den <b>Visdom</b> , bvormed den forstod at benvne fig af deres rige og anvendelige Hjcelpetilder . Det er blevet bemcerket , at Samfundets Seeder for en Tel erc forandrede siden den Tid , faa at de bave gjort Qvindens Betjening as sti eget Kjun mindre nodvendig nu , end det var den Gang , da det venlige og sortrolige Samsnnd mellem Kjonnenc var mere bundet , og <b>kvinderne</b> boldteo i sturrc Ensombed og Afsondring . Ter ligger uden Tvivl nogen Sandhed beri , men maaste ikke saa megen , som Rogle snste at tro . Baade den almindelige og den belligc Historie srcmstiller disse Tiders <b>Qvinder</b> som saadanne , der arbeidede i Samfundet og deltoge i Mandens Beflja ' ftigelser . Jeg bemcorker nu , at Cbristendommen ved sin <b>Aand</b> og sine Forstrister ikke blot saakdcs vil becve den <b>qvindelige</b> Cbaraktcr , men at den ogsaa bar bavt denne Virkning , bvorsomhelst

<h4>137</h4>Skyld , men ved sine Mcrnds Grumhed og Laster , Hvor mange Hustruer bavc ikke levet i et Slags busligt Slaveri , soraarsaget ved Mandens Trukkenskab , Utroskab og Tvranni ; disse Mcend , som dog engang med Ed bavde lovet at elske og beskytte dem . ( shristcudomn , ens Magt bar i utallige Tilfalde erobret og forandret saadannc Ma ' nds Hjerter , luitret og helliget dem , saa at der senere syntes ikke at va > re mindre end en nv 3 Egtefcelle , end et nvt Menneske , et af de storste Beviser paa denne Forandring oar bans forandrede Qpforsel i Hjemmet , hvor Hustruen nu bleo bans Selskab og Ledsagerinde istcdct for bans trcvllende Slaoinde , Saaledes har Christendommcu oed sin <b>Aand</b> og sine Forstrifter udvirket Qvindens Befrielse og Foradling , hvorsomhelst den har vundet Indgang , Middelalderens Ridderiigbed , der oar en Forening as Religion , Tapperbed og Hoflighed , sogte , saa taabelig den end undertiden viste sig , dog i nogen Grad ar naa dette <b>Maal</b> . Jeg vil ikke bestride Sandheden af den Bemcortning , som er gjort af en fransk Forsatter , der siger , at <b>Qvinderne</b> , indespcorrcde i sine befæstede Slotte , civiliserede de Krigere , som soragtedc deres Svaghed , og formildede Lidelsen af de Folelser , som de selv delte . Ter vare dog de , som sormaaede Mandens vilde Lidenskaber og raa Styrke til at udsyre uegennyttige og cedlc Bedrifter og leerte dem at blive menneftelige i sit Mod , den eneste Tvd , som forhen havde vcrrct agtet .

<h4>256</h4>vcere nu gift eller ugift , rig eller fattig , har dog en Kreds , indenfor hvilken hun udMer en <b>vis</b> Magt , enten til Skade eller Gavn , Enhver svinde vil nødvendigvis ved sin Daarstab eller sin Klogskab , sin Dyd eller sine Laster forege sit Folks Vcerdighcd eller paaskynde dets Forfald . Saalcenge <b>qvindelig</b> Dyd blomstrer , beskyttet af det ene og agtet af det andet Kjon , kan et Folk ikke synke altfor dybt . Til en <b>vis</b> Grad er det O.vinden , fom opretholder sit Folks Lykke . Hendes Dyd staar , hvis den er fast og ufordærvet, som en beskyttende Vagt over Folket . Lov , Retfærdighed og Frihed , Kunst og Videnskab , Alt bidrager naturligvis til et Folks Velgaaende , men hvis <b>Kvindens</b> almindelige Sædelighed staar lavt , saa er alt Andet forgjcevcs , medens derimod , hvis <b>qvindelig</b> Dyd og <b>Dannelse</b> er almindelig . Civilisationens Strpm stiger til sin stp ' rste Hckde og bliver frugtbringende for den mest udstrakte Overstade .

<h4>316</h4>Den nnge <b>Quinde</b> bor ikke lades i Uvidenhed om sit Kald . Hendes fremtidige Stilling bor af en <b>vis</b> <b>Moder</b> eller <b>Lcererinde</b> foreholdes bende , for at bun maa ihukomme Nodvendigbedcn af at berede sig for sin tilkommende Vod . Hnn maa paamindes om , hvad der er mere end al lcerdom , at hendes Charalter maa uddannes , ei <b>Maal</b> , som ikte kan opnaaes uden hendes egne Anstrengelser . Hnn maa tidlig lcerc sig selv , at lmu bar et Kald paa Jorden , vcerd al mulig Opmærksomhed og Forberedelse , ikke for at bendcs Forfængelighed derved skal opblccscs , men sor at bendcs Wrekjcerbed om jeg saa tpr sige maa vcekkes , bendcs Kraft oplives og hendes Vlik rettes paa et bestemt <b>Maal</b> . Mine nnge Venner , det vcere Eders <b>Maal</b> , Eders Von , ai I maatte lcere at forståa Eders Kald , ret at berede Eder derfor og med Held at opfylde det ' Ter omkring Eder paa dem , med bvilke I komme i Verorelfe , og giver Agt paa , hvilke der fvncs Eder mest vcerd at tage til Monster . I ville finde Saadanne , bos hvilke Tilbageholdenheden er gaaet over til Utilgjcengcligbed , Virkelysten til Uro , Opmcerkfombcden tit Nvsgjcrrigbed , Munterheden til letsind . Klogskaben til list , yndigheden til Vebagcsvge , Tvgtigbcdcn til Indbilftbed , Ombeden til sygelig Folsombed , Nogle , hvis Kjcerligbed er bleven Skinsyge og hvis Hjcelpsomhcd er gaaet over til en paatrcengende Egensindighed . Vender Eder bort

<h4>429</h4>Opftrsel . Skulde din Sjcel gaa fortabt , saa vilde det ikke vcere nogen Undskyldning for Guds Domstol eller nogen TrD for dig selv i Fortvivlelsens Verden , at du lod dig indtage af Nvillie mod Gudsfrygten ved nogle af dens Vekjenderes urigtige Opforfel . Og nu , ved - Afslutningen af dette Capitel , lader mig opfordre Eder , I Unge , til allerede ved Livets Begyndelse at betcenke det store <b>Maal</b> , for hvilket I af Skaberen sattes ind i Verden . Tomter ikke altfor ho ' it om Eder selv , thi I ere Syndere , I saavel som Alle . I behøve ogsaa I , men I kunne ogsaa vinde den Frelse , som er beredt i lesns Christus . Tomter ikke heller altfor ringe om Eder , thi I ere udødelige Vcesener og kunne arve et evigt Liv . Hcover Eder til Eders sande Vcerdighed ved at hceve Eder til en sand Gudsfrygt , Tro og et Liv i Saudhedcn . Forspilder ikke Eders Liv i smaaligt , om ogsaa uskyldigt Tidsfordriv, uvcerdigt Eders Evner , Eders Vestemmelfe og Eders Pligt . Betragter med klart Blik og fuld <b>Bevidsthed</b> Eders Kald for denne Verden og Eders Vestemmelfe for den tilkommende. Ser , hvorledes et Kald her aabner sig for Eder , hvilket I maa opfylde med en Christens Hellighed , Mre og Velsignelse saavel som med alle en svindes Dyder og Fortrin. Drager Eders Hjerte fra Forfængeligheden , skjenker det til Gudsfrygten . Skjenker Gud Eders Livs Morgen , og da stal dets Dag , den blive lang eller kort , klar eller overskyet , ensom eller tilbragt i Forening med Andre , lys eller mprk , rolig eller stormende , om den end ender ved en bastig Hjemscigelse , saadan som Eders KjM vel kjender , eller om den gaar ned i en lang og mork Aften , den stal dog for Eders salige <b>Aand</b> , befriet fra StMets Byrde , afluses af den skyfrie og evige Morgen , fom ikke efterfølges crf nogen Nat . Da skulle I finde , at <b>Kvindens</b> saavel som Mandens <b>Maal</b> var at forherlige Gud og nyde af hans Vcesen for evig .

<h4>454</h4>for et levende Frugttrce , vorende ved dens Vred ! Ja , uslcre end alt dette ; — thi Ugrcesset , Stenen , Veden kunne dog paa en eller anden Maade anvendes til nyttige Formaal, men den , som selv ikke noget Godt , til hvad Godt duer vel han uden til at vcere en Advarsel for Andre ? Maatte da Eders unge Hjerter banke af ønsket om at gjMe Godt , at vcere nyttige ! Strceber efter den Mre at gjM Godt . Vogter Eder for at nedsynke eller svinde ind i en foragtelig Selviskhed ; udslukker ikke Eders Attråa efter en velgørende Virksomhed , og betragter det blot som et halvt Liv kun at leve for Eder selv . I denne Henseende vilde jeg endog opvcekke Eder til Nidkjcrrhed . Apostelen siger : „ Det er godt altid at vcere nidkar i det Gode . " Nidkjcerhed betyder , som I vide , en inderlig Attråa , som virker en tilsvarende Kraft til at handle i en <b>vis</b> Hensigt og for et eftertragtet <b>Formaal</b> , og naar denne Nidkjcerhed er rettet mod et rigtigt <b>Maal</b> , betegner den en a ' del og opboict Sindsstemning . Dette sindelag fordrer imidlertid stor Forsigtighed i sin Virksomhed , iscer hos de Unge og fremfor Alt hos <b>Qvindcn</b> . Det ligner Ilden , som vioselig kan benyttes til mange nyttige <b>Formaal</b> , naar den anvendes ret og passes , men hvis den forlades uden tilstrcekkelig Bevogtning og Tilsnn , knn foraarfager Ildsvaade . Et varmt Hjerte fordrer et roligt OmdMme for at forbindre saadanne Folger af misledet Nidkjcerhed . Fordi <b>Kvindens</b> Sjccl er mere folsom og aabcn for Indtryk , er bnn ogfaa mere fristet til en nbetcenksom Handlcmaade end Manden og overveier ikke Tingene med den modnere Eftertanke, fom leder bans Trin . Hos Manden holder Forstanden Hjertet i Tomme , indtil det selv er blevet oplyst og overtydet . Hos <b>Kvinden</b> derimod giver Hjertet scedvanlig for efter end Forstauden , og heri ligger Faren for bende til at tage feil af <b>Maalet</b> for sin Nidtjccrbcd og i denne Nidtjccrbcd gaa for langt og handle nden Vesmdelse . Ncesttn alle nyere Tbeorier , vane sig i Medicin , Tbeologi eller i praktiske Gjenstande , finde s Sa ' rdcleoln ' v forst Gebor hos <b>Kvinderne</b> . Altfor ofte mere ledede as sine end af sin Dommetraft , forvildes de ligesom Eva i Paradiset Ved

<h4>464</h4>dom ; thi hvad sagde Herren til Propheten angaaende de hedenske Skikke , som Ilderne efterfulgte ? „ Ser du ikke , hvad de gjFre i luda Stceder og paa Jerusalems Gader ? Værnene opsanke Trce , og Fcedrene optcende en Ild , og <b>kvinderne</b> celte Deig at lave Himmelens Dronning Kager , og Drikoffre for andre Guder " . Hvilken Kappestrid ; alle Sind ere her optagne , alle Hcender ere i Virksomhed , Mcends , <b>Kvinders</b> og Maatte vi drage <b>Lcerdom</b> af dette slette Erempel , og maatte vi <b>vise</b> samme Nidkjcerhed for den sande Gud , som de frafaldne Ipder for de falske ! Christendommen kan ligesaa vel som Hedenskabet afgive en Mark for svinder og VMn til at bearbeide ; og hvor hellige ere ikke de Forpligtelser , som den paalcegger sine Vekjendere om at udbrede den .

<h4>485</h4>Tvivl undertiden varet Fo ' lgen , og det er derfor vigtigt for Alle , som deltage i denne Vestjaftigelse , at vcere faa forsigtige og aarvaagne , at ikke deres Opfprfcl maa give dem , fom fsge derefter , Anledning til at tale ilde om So'ndagsskole-Nndervisningcns Indvirkning paa deres Charakter og Opførsel , fom give sig af dermed . Distriktsforeninger og andre Kjarlighedsindretninger for Afbjalpelfe af <b>aandelig</b> og legemlig Nsd nnder Tilfyn og Ledelfe af svinder ere blevne ganfte almindelige blandt Christne af forfkjelligc kirkelige Samfund . Det var vel at onste , at disse saa lidt som mnligt lededes af yngre <b>Fruentimmer</b> . Den modnere ' Alders Erfarenhed og Anseelse bchp ' vcs for at give faadanne Arbeider deres rette Kraft og Virkning , hvilket dog ikke bindrer , at ved de Mldres Side i disse godeGjcrningcr ogsaa de vngre <b>Qvinder</b> kunne deltage . Pibel- og Missionvselst ' abcr samt andre fromme Stiftelser bave ogsaa aalmet en Mark for <b>qvindelig</b> Virksomhed . Mange <b>qvindelige</b> Hander ere fvsfelsattc med at indfamle og staffe tilveie Midler for disfc vigtige <b>Formaal</b> og forsyne de Fattige med Guds Ord . Herimod kan Intet indvendes , blot det iagttages , at ikke altfor unge Personer anvendes derved . Man bor berved paasc , at Intet sker , som kan vane stridende mod en almindelig Folelse for det Passende og Anstcendige .

<h4>512</h4>som blot er Kjcerlighed til Adspredelser under Religionens Skin . Naar en ung svinde elsker Hjemmet og dets Pligter , men er villig til ved passende Leiligbcd og for et vcerdiqt <b>Formaal</b> at deltage i den ydre religiose Virksomhed og almindelige Velgjorcnbed , da har hnn det rette Sind , Men hvis Hjemmet med dets Pligter forekommer bende kjedcligt , og hun stedse og altid langes ester ydre Virksombed , da er allerede hendes Smag fordcervet , hendes Cbarakter stadet og hendes fremtidige Udsigter til en <b>vis</b> Grad sordnnklcde . Hvis hun har forsaget den Tanke etter det Ansle nogensinde at blive en gift O.vinde , og istedei , ndcn at icenke paa at gaa i Kloster , dog bar besluttet at blive en Barmbjcrtighedssoster, da kan det visselig vcere i sin Orden ganske at hengive sig til velgjorende Virksomhed ; men bvis dette ikke er Tilsceldet , maa hun tage sig i Vare for de Vaner , som maaske engang ville gjore hende uskikket og uduelig til at opfylde Mgtestandens Pligter . Unge O.vindcr bor , saalcenge de forblive i Forcrldrehuset, lade sig lede af sine Forcrldres , iscer sine Modrcs ønsker . De ere ikke , skulle heller ikke unske at vcere uafhcengige af Forcrldrcnes Myndighed . En god og forstandig Datter vil altid med Mrbodigbed og Hengivenhed se op til en god og <b>vis</b> <b>Moder</b> og vil derfor beller ikke begvnde nogen religips Virksomhed uden hendes Raad og Bifald . Det kan hcende , at Barnets Anfker stundom stride mod Forceldrenes Synsmaade angaacnde dette Punkt , det sordrer dog liden Eftertanke at indse og bevise , hvo der i dette Tilfcelde bor give efter . Maaske en ivrig ' og nidkjær ung <b>Qvinde</b> opkaster et Sporgsmaal som dettes „ Jeg foler og anser det for at vcere min Pligt mod Gud at arbeide for Religionens Udbredelfe og mine Medmenneskers Bedste , iscer deres Sjceles Frelse , men mine Forceldrc scette sig derimod : er det da min Pligt at folge min egen Overbevisning eller opgive mit Anste for mine Forceldres ? " Til Svar herpaa vilde jeg sige , at du vel paa en rolig og cerbo ' dig Maade maa tilkjendegive dem dine Onskcr og anfore alle de Grunde , du kan , for at so ' ge at vinde deres Bifald . Men hvis dette fkulde vcere forgjceves , da

<h4>521</h4>Ved Orden og Punktlighed spare og kj / be vi Tiden tilbage , derfor hsrc Orden og Punktlighed til Midlerne til at udfare Kærlighedens og Barmhjertighedens Gjerninger , derved at de forfyne os med Tid til deres Udøvelse . KjM din Tid fra unyttig isning og selv fra den overdrevne Iver for Mnsck , Maling og Pyntarbeider , som er saa betegnende for vor Tid . Vistnok maa ogfaa Saadant Fordring paa nogen Tid inden bedre stillede Familier , thi jeg vil gjerne de Fordele , som Duelighed i nogen af disse Gjenstande kan medfare for en <b>Qvinde</b> . Jeg elsker Alt , som pryder <b>Kvindens</b> Person . Herom vil jeg dog faa Leiligbcd til at tale i et andet Capitel , og derfor vil jcg nn blot spo ' rge , om , naar Np ' dens Raab naa hendes og naar Skabningens Sntke have sig omkring hende , naar utallige Millioner derude leve og do " her paa Jorden nden Evangeliets Lys og Frelsens Haab , naar endog udenfor vore egne Dorc saa Mange henleve sin Tid i Vankundigbed og Synd om det da kan vare en velvillig , ja en cbristen nng <b>Qvinde</b> vandigt at anvende saa Meget af den dvrebare Tid paa sine Arbeider , Noder og Broderier . Kan bun , naar bun sidder over Naalcn og ndvalgcr det formodentlig smagfulde Monster , Time efter Time vare dov for Mennestebedens lammerraab ' „ Kom herover og bjalp os ? " Tanter bun ikte med en eneste Tante paa , hvor- Staver af Sjale erc gaaedc nforberedtc ind i Evigheden, blot siden buu fatte sig ncd sit Mer uden at tale om nogen Virtsombed for ( ibrisii Rige , er det ilte bedrovcligt at se saadannc Pagatcller optage saa megen Tid , som bedre kunde anvendes ti ! Forstandens og Hjertets <b>Uddannelse</b> ved nvtiig og larerig Man kan ikte gjore noget Godt mcd Orden og Plan , nden at indkjobe Tid for dette <b>Formaal</b> . Man tillade mig nu at paaminde b ' dcr om , at alle Eders Bestrabelscr for Gnds Sag bor ste i Troens og VoMnens <b>Aand</b> . Det maa itte blot vare Kjarligbed

<h4>542</h4>uvenligt og deres Vaner uforenelige med buslig Fred , drage de dog snart bort , og Ufreden forsvinder med dem . I forblive derimod , hvis I ikke crc tvungne til at ftge Tjeneste for Eders Opbold , bjcmmc bos Eders Forceldre , indtil I skulle indtrcede i Eders eget Hjem . Hvis I ikke faa noget eget Hjem , forblive I bos Forældrene for bestandig. I dette Tilfcelde blive I Fcedrehjcmmei nlborende , og da er Eders Forpligtelse til at foroge dets Hygge endnu stirre . Hvor megen Fred eller Ufred kan ikke en Datter , alt efter sin Ebaraktcr og sit Sindelag , afstedkomme i en Tid af ti til tyve Aar , regnet fra bcndes Ztolctid ! Derfor er det altid en Kilde , ikke til üblandet Frvd , men ogfaa til Uro og Frygt for en ombyggclig <b>Moder</b> , paa hvilken Maade hendes af Skolen udgangne Datter tommer til at arte sig , naar hun vender tilbage til Hjemmet og efter afsluttede Studier ombytter sin og sine Kammeraters Selskab med Familiekredsen . Her er saaledes det Forske og Vigtigste , som en ung <b>Pige</b> bor opstille som sit <b>Maal</b> ved sin Tilbagekomst til Hjemmet , nemlig saa meget som muligt at bidrage til den huslige Kredses Glcrde og Hygge . Du kan der ikke vcere et Nul , du maa ikke tro , at du kan vcere der , uden ganske at udo ' ve nogen Indflydelse . Det lille Samfunds øvrige Medlemmer maa Alle ftle Indvirkningen af din Opfprsel . Du er altid iblandt dem- dine Handlinger , dine Ord , ja endog dine Blikte indvirke paa dem . Se der Begyndelsespunktet paa de buslige Pligters Bane . Fat da en fast , moden og bestemt Beslutning om med Guds Hjcvlp at gjprc dit Hjem saa lvkkeligt , du kan , for Andre , og derved ogfaa bvggeligt for dig selv . Indprceg denne Beslutning i dit Hjerte , din Hukommelse , din Samvittighed . Er den ikke <b>vis</b> , dydig og ret ? Billiges den ikke af Fornuften, Samvitiigbeden , Hjertet ? Gid den blive for dig en vigtig og alvorlig Gjenstand ; skjent den ikke blot en Tanke , som flvoer gjcnnem Sjælen , uden at lade Spor efter sig , men lceg den dvbt og alvorlig paa Hjertet . Desuden have jo vine Forceldre Ret til at vente sig dette . Er det ikke billigt , at naar du nvder Fordelene af et Hjem , du ogfaa .

<h4>650</h4>som er tilfredsstillet med sit Arbeide , hvorledes det end udftres , blot den betingede Lo ' n opnaacs . Disse <b>Pigers</b> Charakter og Fremtids Vel for tvende Verdener beror for en stor Del paa dig og paa den Maadc , hvorvaa du opfylder dine Pligter . De ere blevne overladte til din Varetogt og se op til dig som sin <b>lcercrinde</b> og sit trempel. Hvis du vil gjore dit sidste Regnskab for Gud med Glcede og ikke med Sorg , gjor da dit Bedste ; anvend alle dine Krcefter og Evner paa at uddanne deres Charakter , udruste deres Forstand med Kuudskaber og paa enhver Maadc berede dem for den Plads , som de crc bestemte til at indtage i Livet . Bered dig selv til dette Arbeide ved fortsat Lcesning og Selvstudium , vcrr itkc tilfredsstillet med den Dygtigbed , du allerede besidder . Et fornuftigt og udødeligt Vcesens Opdragelse for denne og den tilkommende Verden er et vigtigt Vcerk . Veflu dig fclv paa <b>aandelig</b> Udvikling for at kunne befordre Andres . Sog at vinde deres Tiltro ved din Dncligbed , deres Hcngivenbed ved din Venlighed , deres Agtelse ved din Vcerdigbed og deres lcervilligbed ved din egen Flld . Lad dit <b>Maal</b> vcere at meddele ikke blot <b>Kundskaber</b> , men ogsaa <b>Visdom</b> , ikke blot at belosse Hukommelsen , men ogsaa at ove Dommekraften , stvrke Villien og vcette Samvittigbcden , at lcerc dem ikke blot at tcenke rigtig , men ogsaa at handle ret og samvittigbedssuldt, i bvilkcn livsstilling de end komme til at indtrcrde .

<h4>716</h4>Ceremonier , Kulde og Afmnalthed erc her borte . Den ungdommelige Onde af Naturens oprindelige Renhed , Sandhed og finhed trcedcr atter for os , aaben og ukunstlet . Vi indaande en renere og friere Luft . Vi bevcegcs paa engang af en dybere Folelsc af et foerdelcs Himmelens Forfyn og en virkelig og fand paa Jorden . Vi ftlc , at der kan findes noget Saadant fom en barnlig og « del Tillid baade til Gud og Mennesker , og at det er muligt at handle i Overensstemmelse med denne Tro paa Guds Ledsagelse og Menneskers Redelighed . Forinden vi betragte den <b>Lcerdom</b> , som kan vindes af Rebekkas Opftrfcl som Wgtefcrlle og <b>Moder</b> , ville vi for nogle blive staaende ved bende med farligt Hensyn tll den Begivenhed , der gjorde bende til dette , nemlig bendeo Giftermaal . De Omstændigheder , som dermed vare forenede , vare ciendommeligc for disse Tider og frembyde en Simpelhed, hvortil , fom jeg allerede har bemcerket , den nyere Historie ei kan opvise et Sidestykke . En Ting maa dog bemcerkes , hun giftede sig med sine Paarorendes Vidende og Samtykke . At hendes Fader Betbnel med Tausbcd forbigaaes, kan blot forklares faaledcs , at ban itte mere levede . Men Laban raadspurgtes . Her vare ingen Hen , - meligheder med i Spillet , og desuden var det et Giftermaal, ved hvilket Religionens Fordringer til en <b>vis</b> Grad haodes for M . Hvis Rebekka viste altfor stor Iver med at indtråde r Wgtestab , hvis , ' buu altfor hastig fattede sin Beslutning , vil jeg vist ikke billige eller anbefale noget Saadant . Jeg bar dog allerede fritaget hende for denne Bestvtdmng . Tet tan vcrre naturligt ' nok at foretrcekke den gifte fremfor den ugifte Stand , uaar Anledning tilbvder sig til at indtrccde deri . Man bor dog ei lade sil Hjerte ncere en altfor utilbørlig og urolig Lcengfel efter at faa sine Dufter virkeliggjorte . Man bor aldeles ikke lade Indbildningen faa frit Spillerum i denne vigtige Sag . Tillad dig ei at behandle den med Letsind , <b>vis</b> ei en Opforsel , fom kunde forraade , at du var mere begjærlig efter at indtrcede i Mgtcstandcn end at blive ret stikket for dens Pligter . Giv ei Plads for den Antagelse ,

<h4>760</h4>Der staar endnu tilbage nogle Lårdomme , som en <b>Qvinde</b> har at hente af Rebekkas Opftrfel under sit fencre Liv , Landomme at medbringe og ihukomme under hele Livet . Den fyrste er en almindeligere : forandrede Omstcrndighcdcr medfare ofte forandret Cbarakter og forandret Opførsel. Hvor forskjellig var ikke her Jomfruen i Mesopotamien og <b>Husmoderen</b> i Cana ' an ! Og er det vel noget Usædvanligt ogsaa nu at se Forandringer , langt mere omfattende og gjcnnemgribcnde , fplge paa en Omvindes Indtrædelse i Wgtcstanden ? Betragt videre denne enkelte Lcerdom, at vi aldrig bpr arbeide for et godt <b>Formaal</b> med slette Midler , eller med andre Ord , ikke gjore det Onde , for at deraf maa folge det Gode . Affty denne lesuitcrnes Grundfcetning og Ondlingssats , at „ Hensigten helliger Midlet/'og affty i Særdeleshed at fxilgc denne ugudelige Grundfcetning i det , som angaar Religionen . Hvilke Forbrydelser ere ei i Religionens misbrugte Ravn blevne udpvede af romerske Ivrere for deres Kirkes Bedste ! Og ogfaa af Andre , i Navnet Christne , med renere Tro og bchcelede af en mildere <b>Aand</b> , hvor Meget er ci blevet gjort , som det hedte , for Religionen , men i Virkeligheden tun for at befordre sceregne Hensigter , i Strid med Alt , som Guds Lov og Kjoerlighed til Nersten byder . Religionen vil ei betjenes af nogen anden <b>Aand</b> og bcvcegende Kraft end sin egen ; den foragter alle Tilbud , som stride mod Sandhed , Kjcerlighed , Redelighed og Wre . Og jo stprre vor Nidkjcerhed for en Sag er , desto mere ere

<h4>779</h4>hos den Ene er Fornuften hos den Anden . Philosovhen er af Jord , jordisk : Apostelen taler i Guds Kraft og lader os ftlc den usynlige Verdens Magt . Medens Plato leder <b>Qvindcn</b> til udelukkende at soge den Wre , som kommer sra Mennesker , lcercr Petrus bende at ftge Guds , den alene Fuldkomnes , den alene Herliges Wre og Pris . Forinden jeg forlader denne ' Gjenstand , vil jeg just paapege den Mune Maade , hvorpaa Apostelen henviser til den Del af vort Vcescn , som det burde vcere vor fornemste Omsorg at pryde . - „ Det skal vcere , " siger ban , „ Hjertets skjulte Mennneste i en sagtmodig og stille Aands uforkrænkelige Vcescn , hvilket er meget kosteligt for Gud . " Hvor Mn er ikke denne Fremstilling , hvilket Dyb af Tanter ligger ei i disse faa Ord ! Det er Sjcelens og ikke Legemets Prydelse , som iscer bor lcegge Beslag paa den christne Qvindes Opmcrrksombcd . Hvilke ere de Prydelser , som anstaa Sjcelen ? Sjcelen er udpdelig og uforgjengelig ; saaledcs bM også « dens Prydelser vwre . Hvorledes kunne Guld og Juveler passe hertil ? Kan vel Diamanten oplyse eller overstraale Fornuftens Lys ? Eller Rubiner forboie Hjertets Unde ? Etter den crgte Perle indfattes i Samvittigheden? Eller en straalendc Tragt klcedc Charaktcrcn ? Eller Fjedre og Blomster vaie over en ny og himmelsk Natur ? Nei , Sjattens Prydelser ere Sandhed , Hellighed, Tro , Haab , Kjceiligbed , Ydmyghed , <b>Visdom</b> , Glcede , Sagtmodighed og Mildhed ; disse ere de Juveler , hvormed det indvortes Menneske burde pryde sig . Det udvortes, der jordiske er forgjcengeligt . Swv er det , StM skal det atter blive . Hin Skj.onhed , som nu straalcr i Juveler og Mdclstcne , en Prydelse for Balsalen og en Gjenstand for Avind , vil inden kort vcrre en Masse af Forraadnelfc, altfor gruopvattkende til at beskue , siden et vederstygeligt Skelet , en Samling af Benknogler , en Haandfuld StM , og bvor er da Sjcelen den udødelige <b>Aand</b> ? Mon den fkal bcere Legemets aflagte Pynt ? O nci , de have maaskc endnu undgaaet Forgjcengeligheden , dog blot for at pryde andre til Dpden hjemfaldne Legemer , men ei til at berede Sjcele for evig Herlighed . Men tcenk nu paa dm

<h4>781</h4>Qvinde , som , uden at legge altfor stor Vegt paa Legemets Udstyrelse , og fremst efterstræbte <b>Sjelens</b> Prydelser ; se hende , som var iftrt Retfærdighedens Dragt og Salighedens Kledebon , prydet med en ydmyg og stille <b>Aand</b> . Ogsaa hun og det <b>Maal</b> af jordist SkjMhed , hun havde , hjemfalder , ogfaa det , for en Tid til Forgængeligheden; men hendes udpdelige og uforgjengelige <b>Aand</b> , over hvilten Doden ci har nogen Magt , fremtræder ei uden Prydelse for den Eviges Ansigt ; thi de Juveler , hvormed den ftgte at pryde sig paa Jorden , ere ligesaa uforgjengelige, som den er selv , og de frcmtrede med den for at straale i Guds Ome ; thi fra ham udgik deres Glands , til ham vender den atter tilbage . „ Menneskene anfe det usselt og nedværdigende at verc ydmyg . Intet er mere udfat for Foragt end en sagtmodig <b>Aand</b> , den er Daarstab for Mennesker ; men hvad betvder det : mere end al Vauerc gjelder vel det , at den for Gud er HM agtet . Og Tingene ere jo just saadaune , som han vurderer dem , og ci anderledes . Lad at det ci er Landets Sedvane , det dog Sedvanen ved Hoffet , ja , det er selve Kongens , vor Konges egen Sedvane : „ Lerer af mig , " siger han , „ thi jeg er fagtmodig og ydmyg af Hjertet . " Man ved , at de , fom ere opdragne ved Hoffet , pleie at forskaffe sig sine <b>Moder</b> derfra , selv om de ei bo der ; Vondernc kunne synes , at de se besynderlig ud i sine Dragter , de agte det dog ikke , men antage , at Hoffets Dragt er den bedste og smukkeste . Gjor ogsaa du saalcdes , bryd dig ci om , hvad Vcrdcn siger : du stal jo ikke forblive her faa lenge . Skaf dig baadc To ' i og Snit fra Himmelens Hof . ydmyghedens Dragt og sagtmodighedens kledning stal da sendes dig . Og ber disse Meder for bans Skyld , fom sender dig dem . Han bar sin Lyst i at fe dig dem ifo ' rt ; er dette ci not ? Vi kunne i Intet tomme til dct Rette , forinden vi have lert os at foragte Mcnnesters Omdomme og blot rette vort Blik paa Guds

<h4>817</h4>kostbare Perler . Men bvis alt dette er vigtigt for en ugift <b>Qvinde</b> , bvor meget mere forogei bliver ilte dets for den , fom agter at blive A ' gtefeelle og <b>Moder</b> ! « vor kan du vel uden Forstand og Smag vane stikket til at blive eu <b>vis</b> og forstandig Mands ledsagerinde eller med Hel o at vaage over dine Vorns Dpdragelfe ? Veternt , i bvillen du lever . Men ogsaa i disfe <b>Dannelsens</b> og Smagens Dage , vi vide det meget godt , er en god Husmoders Dvgtigbee uvurderlig ; thi der ligger liden Verommelse i det Omdomme , at en

<h4>888</h4>bestandig grublende over , hvad der klceder og trcesser Smagen - Nogle tcenkc blot paa Fornyelser og Gjcrstebud og drMmi altid om en Anordning til at glimre og valke Opsigt- Andre atter kunne ei blive forsynlige nok og bestrcebe sig blot for at spare saa meget som muligt . Paa alle disse Maader kan en svinde plage sig sely og fylde sit Sind med unMg og felvpaalagt Bekymring , Til alle Saadanne sige vi : „ Martha , Martha ! Du bckvmrcr diq og forstyrres ved mange Ting " . ' ' ^ Foren derfor Marias brceudende Gudsfrygt med de bedre Egenskaber hos Martha , hendes Dugtigbed og Flid i Husholdningen . Skulde du ville give dig tilfreds med de Egenskaber , fom blot gjore dig duelig for den Bolig , hvorfra du mden en Time kan vare kaldet heden ? Bcor ligefaa omhyggelig som Martha i de huslige Sysler , men var derhos ligesaa „ brandende i Aanden , tjenende Herren " ( Rom . 12 , 11 ) , som Maria var . Spg at forene alle en Helligs Tyder med alt det , fom anstaar en Wgtcfalle , <b>Moder</b> og <b>Husmoder</b> . Var Alt , hvad du skal vare , i dit eget Hus og Ali hvad du bor vare , i Guds Hus , det er i de som tillMe ham . Opfnld en dig maaste bestemt Wgtcfalles Forhåbninger; kom bans ønsker impdc . Tetie bor dn , det maa du . Hvad Christus ønsker og begjcercr , er ' at fe dig Ndde ved hans Wdder og hMe hans Ord . Kom ogsaa dette Onste inip ' de thi ogsaa dette er din übetingede Pligt . O I , som trykkes af Bekymring og Omtanke for disse mange Ting , lader der blive en Ende paa Eders Sorg . „ Vcercr ikke bekymrede for Noget , men i alle Ting lader Eders Begæringer frcmfMs for Gud i Paakaldclse og Bon med Taksigelse " . En sand Gudsfrygts <b>Aand</b> stal mildne det huslige Livs Sorger , lette Bekvmringen og stjenkc en Glade , som vil faute Eder istand til frimodig at forrette de buslige Gjerninger , og en ombvggelig Agivaagivenbed paa Hjemmets Pligter vil , langtfra at gjore Eter ufMede for Omgangen , med Gud , tvcertimod give Eder Noget at bede om i tilstrækkeligt <b>Maal</b> og blive for Eders Sjcel en tvingende An- - ledning til at soge Herrens Ansigt . Og nu lader rnig til Slutning gjore Eder opmærksomme

<h4>934</h4>al se ct Hus , hvor Alt syncs at vcere OprFr , hvor Ordenen er borte , den faderlige Myndighed brudt , Moderen foragtet og vanartige VM i Besiddelse af Magten . Og dog maa en Fader saavel som en <b>Moder</b> adlydes , og det er hendes egen Feil , hvis hun ei er det . En <b>vis</b> og fast Styrelse , hvor Kjcerlighedcns Haand holder TMcn , vil sikkerlig frembringe Underdanighed til Slutning . Men det maa vcrre en Blanding af Kjcerligbed , <b>Visdom</b> og Myndigbed. Nndcrdanigbeden maa kjendes af Varnet og erkjendes som en Pligt og ikte blot vindes som en Hengivenhedens Tribut . Myndigheden maa ei henstivnc til Haardhed , ct heller Kærligheden til blotte Kjcortegn . Befalingerne bM adlydes , ei blot fordi det er en Tilfredsstillelse , men fordi det er en Pligt .

<h4>963</h4>Vehsivcr jeg at sige , at i Alt , hvad du gjor i og for dine Wrns Opdragelse , bpr du altid , direkt eller indirekt, rage Hensyn til deres evige Velfcerd ? Skulde jeg ogsaa synes at komme tilbage til dette Emne altfor ofte , saa maa dets evighedstunge Vigtighed og dets desuagtet altfor sædvanlige Forftmmelse vcerc nun Undskyldniug . I Betragtning af at dine Vorn erc bestemte for en Evighed og skabte med Evne til at nyde Himmelens Salighed , bjK du allerede fra Barndommen af soge at vcekke hos dem Hu og Siud for det Aandelige og Evige . Det er Evigheden , som med sin Ncervcerelse det Lille og Übetydelige stort og vigtigt, og det er ogsaa deras , at en Moders opboicde Værdighed i en ikke ringe Grad er betinget . Hun har paa den Almægtiges Bud givet Livet , ikke til et Vcescn , som skal dp og forgaa lig Dyrene paa Marken , men til et udødeligt ! Hendes diende Barn , svagt og hjcelpel.cist , som det nu ser ud , bcerer dog i sig en fornuftig Sjcel , Evue til at fole og tcenke , et Liv , som ingen Tid tan forstyrre . Himle og Jord skulle forgaa med sin Pragt , men det fattigste , det ringeste, det usleste Barn har en <b>Aand</b> , som skal leve i alle Evigbcdcr . <b>Moder</b> , dette er din Vcerdigbcd . Fol og vurder den Udmcerkelsc , dig sker , naar du skal kaldes til at opdrage den almcegtige Herres Sønner og Dsttrc og at forberede Medlemmerne af den hellige Familie , som skal bo i de mange Boliger i Faderhusct , hvor Jesus er gaaet hen for at berede dem Sted . Hengiv dig med Hjerte og Sjcel , med alle dine Evner til dette cercfnlde Vcerk . Men vcer i ' Alt forsigtig og bctcenisom , saa at du ei snarere opvcekker Uvillie og Afsky end Lyst og Tilboiclighed sor den sande Gudsfrygt . dig Kjcerlighcd , Mildhed og Glcede , naar du uuderviser dit Barn i selve <b>Visdommens</b> Begyndelse . Lev og vcer for dem jnst saalcdes , som du lauer , at Herren byder . Lad dem lcere af din Cbarakter og Handlemaadc ligesaa vel som af dine Ord . Og vcer iscer omhyggelig i ikke at frembolde fom et Lovbud det , som blot burde vcere en Kilde til Gloeve , en Gjenstand for Haab . Lad dine

<h4>1022</h4>I forrige Capitel gav jeg nnge <b>Modrc</b> nogle Anvisninger om deres VFrns livligste Opdragelse ' , jeg vil nu give dem nogle Vink , bvis Vorn opvore omkring dem eller allerede cre blevne Unglinger og Jomfruer . Jeg siger da til saadannc <b>Modre</b> : Holger Eder gjerne bjcnnne , og saa vil det med en iovrigt venlig og kjccrlig Vebandlingsmaade lvkkcs Eder at bolde dem bjemme . Gjorer Eders Selskab bebageligt for dem ved at lade dem se , ai deres Omgængelse er Eders Glcede . Spreder Glecdens og Hvggens Skin over den buslige Kreds . Vcrrcr selv glade og tilfredse , og Vornene omkring Eder ville ogsaa fole sig lvkkcligc . Men det maa dog ikke vcrrc en Munierbcd , som udarter til sindigbcd . Intet er mere utilladeligt end en tankclos og verdslig <b>Moder</b> , som tillader npassende Tale eller tomt Sladder eller lost Snak at foregaa i sin Ncrvvcerelfe mellem Vornene . At kaldes en „ Sladdcrsoster " er ikke anbefalende for Nogen , allermindst for en <b>Moder</b> . Og af ali Skjemt , lom en forstandig og ombvggclig <b>Moder</b> skal undvige , cre Forelskell ' er og Kjccrligbcdseventvr det , som der mindst bor vecre Tale om . For sine Sonner bor bun vcere det Vlvnstcr, efter bvilket bun vilde se dem vcelge sig Hustruer , for sine Toitrc det Ereinpel , bun onfkcr , de vilde folge , bvis de nogensinde selv blive Hustrncr og <b>Modre</b> . Hendes Holdning bor veerc boi og vcerdig , nuen formildet ved en Kjcerligbedcns Aandc ; og den Venligbed og Oprigtigbcd , fom bersker , vcerc dog fjern fra en i Siovct ncddragcnde Fortroligbcd . Hendes Mvndigbed bor fo ' lges af oicblitkclig Vvdigbcd , af bvad Alder bcndes Vorn end maa vcrrc- , bendeo <b>Visdom</b> bor vinde deres Tillid , bcndes Kjcerligbed deres Gjenkscerligbed og Takncmmcligbcd . Huu maa faaledeo veere deres Selfkab , Raadgiver og Trost og ved sin egen Oprigiigbed opmuntre deres . Te maa bebandles faalcdes , at de ikke onfke at bave vigtigere Hemmeligbeder for bcudc . Iscer burde bun i sit Vcesen <b>vise</b> dem al den sande Gudsfrvgls Rcnbed , Odmvgbcd , Fastbed , Skjoubcd og Onde , dens renscndc , vdmygende og helliggørende Kraft i Medgangen

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009072703011'target=_>Ahlfeld, Friedrich, 1863, Menneskelivet</a><h4>403</h4>de Vtnnyge giver han Naade " ( iPed . 5 ) . Men da Mrgjerrigheden ikke ligefrem griber ind i andre Menneskers Rettigheder og Eiendom , har den sundet Naade for Verdens Bine og er bleven hedret med mange stjonne Navne . Ja , man har mange Gange brugt den og bruger den endnu som eu Hjelper ved Opdragelseu . Hvad Sporerne ere for Hestene , det har 3Ergjerrigheden ofte manttet vere for Bornene ; mange Born have drevet sig selv frem dermed , idet de have feet hen til dem , der bere store Navne i Verden , og have bestrebt sig for at nnne dem eller overgaae dem . De hnve lignet hiin greske yngling , som sagde : „ Miltiades ' s Bedrifter lade mig itke Ro til at fove " . Men ofte begaae Foreldrene felv den Feil at gjore Mrgjerrigheden til en Hoveddrivefjeder for deres Born . De formane dem til ikte at lade sig overgaae af nogen Anden , ja de fremstille det forn en Stam , nåar dette steer . De anspore Barnet ved at sige : „ See , at Du kan overgaae den eller den " . De rose Bornene ved ethvert Fremstridt , ved ethvert godt Vidnesbyrd , jn de stille det ene Barn ved Siden af det andet , fammenligne dem med hverandre og fremheve det ene forn et Monster for det andet . Der tales om at staae til Skamme ved Examen , og Bornene opmuntres til at gjore sig og deres Foreldre 3 Ere . Det er bestandig Mrgjerrigheden , der stal anspore dem ; men netop dette er en fnlst Drivefjeder . — Og forn Foreldrene , fnaledes bere ogsaa Lererne sig ad . De forste Plndser i Stolen mnac ofte tjene forn Lsftestenger for Fliden ; de bedste Charakterer maae vekke og anspore Arbeidslysten . Mange <b>Modre</b> begynde nesten regelmessig ! med folgeode Setning , nåar de omtale deres Barn : „ Han har altid havt de bedste Vidnesbyrd i Skolen , han har altid bragt gode Charatterer hjem " . Ja , man er gaaet saa vidt , at man har maalt dem med det samme <b>Maal</b> ved Consirmationen og bedomt dem efter deres Flid , deres Nummer og Vidnesbyrd i Skolen . — „ Men " , sige I , „ stal man da ikke rose et Varn ? Stal man itte <b>vise</b> det Anertjendelse , nanr de ! har arbeidet trofast ? " Kjere Christne , Roes og Mre stnlle i det Holeste vere Krydderi paa Maden , men aldrig Mnden selv . Vee det Barn , hvis Strebsomhed beroer paa Andres Roes ! Vee det Barn , som itke foler en indre Drift i sig selv til at opfylde sine Pligter ! Det bliver et

<h4>592</h4>Med losephs Liv for Aie ville vi forst betente , hvorledes den modnere Inglingsalder ftal benyttes . Hvorledes benyttede lofeph den ? — Denne Angling var sytten Aar gammel , da han blev solgt af sine Brodre til Mgypten , og omtrent i den samme Alder ere vore Born , nnar de have gjennemgaaet Lerennrene og trede ud i Livet . Meu Ingeu gnner snn fattig ud i Verden , Ingen kommer i faa streng en Tjeneste som Joseph . I blive Svende , Medhjelpere eller noget Lignende ; men han blev Slave , han tom til at staae udenfor Lovens Bestyttelfe . Han traadte ind i en heteust Wgypters Huus , hvor der intet Spor var af Gudsfrygt . Ingen Fader tuute ledfnge ham med sin Formaning , intet Brev , som kunde hnve styrtet ham og holdt ham oppe , indlob til ham . Ja , ingen Bon steg op til Guds Throue for ham ; thi hnns <b>Moder</b> lna i Graven , og hans Fader gred vel over hnm , meu bad itte for hnm , hnn stod jo i den Tro , nt et vildt Dyr havde souderrevet ham . Mere forladt tan Ingen vere , end denne L ) ngling var det i ZEgypten . Og dog betrngtede hnn sin Stilling som en Tilstittelse nf Gud , dog beuyttede hnn den til en erlig , dygtig <b>Uddannelse</b> , og dn navnlig til den Uddannelse, der fremfor Alt er nodvendig . Derfor arbeidede han mod Stoltheden og Forfengeligheten , der vilde styde op i <b>Sjelen</b> ligesom det pralende Utrudt , derfor tlyngede han sig barnligt og tet op til sin Gud . Ogsaa som Slave vilde han vere Guds Tjeuer ; i det fremmede Land vilde han holde rigtig fast ved sin Arveret til Herrens Forjettelser . Hvorledes han i sin Fornedrelse stred en god Strid mod Fristelsen til Ukydsthet » , er Eder noksom bekjendt . Men han benyttede ogsaa Tiden til at staffe sig sit Udkomne og tomme frem i Livet paa en erlig Maade . „ loseph blev en lykkelig Mand , " hedder det i Textcn , det vil sige en Mand , for hvem Herren lod Alt lyktes ; men

<h4>643</h4>Pige har lavet Pynt , Blomster eller Cigarer , nåar hun har stanet ved en Hurtigpresse eg lagt Arkene ind eg tåget dem ud igjen , nåar hun har passet en Spindemastine eller paa anden Maade har veret beskjeftiget med et eensidigt og eensformigt Arbeide Dag ud og Dag ind ? I mene at have storre Forteel af denne Levemaade ; men det er itte sandt . I gaae glip af Hjemmets stille Gleder , af den bedste <b>Uddannelse</b> til at blive en flint <b>Huusmoder</b> , forn netop beroer paa det stadige , tilvante Arbeide , jn , I gane ofte tnbt af meget Mere ! Naar Eders eget Huus siden bliver Eder for stille , nåar I hverken have Lyst eller Evne til at besorge dets mange fmaa Arbeider , nåar I itte ere det Kald voxne at forestaae en jevn Hnusholdning , nåar den egtestabelige Lytte strander paa disse Klipper : san ville I engang erkjende, hvilken Skade den tidlige Selvstendighet » og Gevinst har forvoldt Eder . Fordi Joseph havde tjent , kunde han siden herske ; fordi han havde lert at forvalte fremmed Gods , tunte han forvalte sit eget . Men det Bedste , som han medbragte fra sin Tjenestetid i Potiphars Huus og i Faugenstabet, var V ^ myghed . Og betragter nu vort Brev . Onesimus var Tjener , Slave hos Philemon , en Christen i Colosse i Asien . Det behagede ham ikke i hans Herres Huus , og hnn flygtede bort . Skriften melder Intet om , hvad han har manttet udstnne paa sin Flugt , om det er gaaet ham godt eller dnnrligt ; men det er dog vistnok ganet hnm dnarligt ; thi en faadcm bortflygtet Slave var jo dengcmg som et vildt Dyr , der var undstuppet af Buret , Jegerne med hele deres Kobbel vare efter ham . Onesimus flygter til Rom . I den store Vrimmel af to Millioner Mennester tan en Flygtning lettest skjule sig ; men vistnok tan ogsaa en <b>Sjel</b> lettest gane tnbt her , hvis Herren ellers endnu hnr Deel og Lod i den . Og Herren havde endnu Deel i Ouesimus ' s <b>Sjel</b> , ja , han vilde have den heel og ganske . Han ledede ham paa hans vildsomme Vei til den fangne Paulus . Men Paulus bringer ham ved sin megtigc Prediken, megtig felv i Lenker , til den Herre Jesus , der alene lan gjore fri . Kun den , forn Eonnen frigjor , er virkelig fri ( Joh . 8 , 36 ) . Onesimus erkjendte , at den Frihet » , som han havde opnaaet ved List og Bedrageri , vnr en falsk Frihed . Han lod sig vinde for Guds Boms falige Frihed , og den blev

<h4>733</h4>itte stemnete overeens , og jeg da har spurgt dem , hvorlenge de havde tjendt hinanden for Forlovelfen , har jeg hyppigt faaet det Svar : „ Fire Uger , sex Uger , otte Uger " , ja undertiden sagde de vel endog : „ Vi have i Grunden slet itke tjendt hinanden rigtigt iforveien ; vi have fjeldent truffet fammen . " Spurgte man dem videre , om de nogensinde havde talt med hinanden om deres christelige Tro og Haab , om de havde bedet med hinanden , om de vare gaaede til Alters sammen , saa lod Svaret „ Nei " og atter „ Nei " . Hvor stal da Enigheten tomme fra . Forlovelser , som sluttes i tjotelig Lyst og af verdslig Beregning , dele Kjodets Lod . Alt Kjod er Gres , al dets Herlighet , forn Martens Blomst ( Es . 40 , 6 ) . Unge <b>Piger</b> , som itte fole nogen Glede over et stille Hjem , hvem Balsalen er tjerere end Stuen og Kirten , forn gaae ud med Dina for at fee paa Hedningenes Dottre , blive aldrig rigtige <b>Huusmodre</b> . Herrens Huus og hendes eget Hjem bor vere din Brud de tjereste Steder . Kun nnnr en <b>Pige</b> tan sige til Dig , som Ruth fordum fagte til sin Svigermoder : „ Hvor Du gaacr hen , vil jeg gaae ; hvor Du tager Herberg , vil jeg blive ; dit Folk stal blive mit Folk , og din Gud min Gud " , saa kan hun blive din Brud , saa kan Du vere <b>vis</b> pnn , nt der vil herske Enighed imellem Eder . — At nu en snndnn Forbindelse itte tan stuttes udeu Foreldrenes Velsignelse , at de fremfor alle Andre maae cidfporges , og at der maa bedes om deres Velsignelfe , det burde staae fast i enhver Christens Hjerte . Det Samtykte , som afnodes dem efter Forlovelfen , er ingen Velsignelse . Du bor agte deres Mening saa hoit , at Du endogsaa boier Dig for deres Nei , og vil Tu <b>vise</b> dem den rette LErefrygt , faa raadfsrer Du Dig allerede med dem , faafnart Tilboielighedeu begynder at fpire i dit Hjerte , forend Du endnu har gjort et eneste bevidst Stridt mod dit <b>Maal</b> . — Til disse forste Fordringer slutte ! , g nu andre . Mgtefolt bor ogfaa med Hensyn til Alderen staae i det rette Forhold til hinanden . Det har sjeldent viist sig vel gjort , nåar Naglingen har ratt sin Haand til en meget eldre Brud . — Den <b>Hustru</b> , som Manden velger sig , maa tunne forestaae sit Huus godt ; den Mand , forn retter en <b>Pige</b> sin Haand , maa tnnne rogte et erligt Kald eg crnere sin Familie ved sit Arbeide . Det er eu yntelig Mand ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016101748015'target=_>Calman, A.L., 1876, John Ashworths Liv og Virksomhed</a><h4>601</h4>med det Gode se til at faa ham til at gaa did , og hvis man blot tager Sagen fat paa den rette Maade , vil man nok se , at man faar sin Hensigt opnaaet . Saa raader jeg ogsaa indstændigen til , at man begynder at lcese den hellige Skrift hjemme i Husene . Dette bringer mig til den sidste Del af det , som jeg onster i nærværende Foredrag at komme til , nemlig Moore . Det er den allervigtigste Del , og jeg maa tilstaa , at jeg aldrig kommer det noer , uden med Mrefrygt og Mistillid til mig selv . Thi det er ingenlunde en let Sag at raade med Hensyn til Opdragelsen af Born , og i mange Familier finder man Karaktererne saa forskjellige . Det , som vilde virke fuldstændig knusende for et Barn , vilde neppe gjore det mindste Indtryk paa et andet . Hvilken <b>Visdom</b> , hvilken Dommekraft , hvilken " Fasthed fordres derfor ikke ? Hvis der er nogen paa Jordens Overflade , som treenger Herrens scerlige Bistand, saa er det Moderen til en Borneskare . Og hvilken deilig Lyd ligger der ikke i Modersnavnet ? Der er tre Ord : <b>Moder</b> , Hjem og Himmel — hvilken himmelsk , hvilken velsignet Lyd de give ! Hust paa , at aldrig nogen Nation hæver sig over sine <b>Modre</b> ! Hver Nation bliver nemlig netop til hvad dens <b>Modre</b> gjor den til . Gud har nemlig lagt et Kjærligheds Dyb i Dit Hjerte , Marie , som kun staar tilbage for Hans egen Kjærlighed . Og Han har givet Dig en Magt , som aldrig kan bestrides med Ord . Naar derfor et Barn bliver lagt i Dine Arme , Marie , saa stal der fra Dine Lceber udgaa Ord , der i den lille übodelige Sjæl vil <b>vise</b> sig virksom i god eller ond Retning for Tid og for Evighed . Lamartine har sagt : „ Den Haand . der passer Vuggen, behersker Verden ! " og det er sandt . Voer derfor

<h4>707</h4>Efter noget mere end ti Maaneders anstrengende og uafladelige Arbeide for Evangeliets forkyndelse og Maadeholdssagens Fremme , gjor zeg nu mine sidste Forberedelser til min anden Afreise til de Forenede Stater . Naar jeg nu stiller de to Gange sammen , da jeg drog over , — først i 1867 , da jeg drog afsted uden en Ven i Verden at sige Farvel til , foruden Dem og Hr. W . , da jeg reiste til et Sted , jeg intet vidste om , og var et rent Offer for en Lidenskab , der næsten havde medført fuldkommen Undergang for mig , da jeg ikte brod mig stort om , enten jeg levede eller jeg dode , da jeg var uden Gud og uden Haab i denne Verden, og såa nu i 1874 , da jeg er ordentlig klædt og mit Sind er vendt mod det rette <b>Maal</b> , da Hundreder af arme drikfældige Mænd takke mig for , hvad jeg har talt og vist dem , nu da mangen arm <b>Hustru</b> , mangt et stakkels Barn , som for ikke længe siden vare elendige , ere gjorte lykkelige og hjertelig takke Gud for de Ord fra mine Lcrber , som formedelst Bistand af Herrens <b>Aand</b> fik en saadan Forandring til

<h4>941</h4>og sin Ironi — jublede , da de hørte Dommen . Han vendte rolig tilbage til sit Fcengsel , og i Kritons, Platons og mange af sine groedende Disciples Nærværelse tsmte han Giftbcegeret . Da Kriton spurgte ham , hvor han snstede at blibe begraven , saa svarede han smilende , uagtet Giften hnrtigt fuldførte sit Odelæggelsesvcerk : „ Du kan begrave mit Legeme , hvor Du behager, Kriton , men Du kan ikke begrave mig . Sokrates vil stedse leve . " Da man senere begyndte at paafkjsnne hans <b>Visdom</b> , Godhed og andre store Egenskaber , angrede Groekerne , at de havde dsmt en slig Mand til Døden; Anklagerne og Dommerne , der havde fordsmt ham , bleve kastede i Fcengsel eller maatte flygte til andre Lande ; man forfattede Sange og Lovtaler over ham og reiste en Billedststte til hans Mre . Grcekerne drcebte ofte først sine store Mænd og dyrkede dem siden . De fleste Skribenter omtale Sokrates som en vittig , taalmodig , velvillig Maud , og nogle kalde ham „ den Attiske Spogefugl " . Hans <b>Hustru</b> , Xantippe, satte ofte hans Taalmodighed paa Prove . I et af hendes ustyrlige , lidenskabelige Udbrud udstjældte hun ham alt , hvad hun formanede . Da hun saa ham taalmodig og munter stelle sig til at gaa ud , lob hun opad Trapperne og hceldte et helt Spand Vand over ham , idet han gik henad Gaden tæt under Vinduet . Men Sokrates smilede blot og sagde : „ Jeg var <b>vis</b> paa , at der vilde komme Regn efter stig stærk Torden . " Engang havde man indbudt ham til en stor Fest , og da han saa alle de kostbare Retter , som vare fremsatte paa Bordet , ytrede han :

<h4>1047</h4>over sine Medmennesker ; det slette Menneske vil aldrig blive et godt Menneske , for han bliver gjenfod t . Hav det til <b>Formaal</b> i alle Eders <b>Moder</b> og alle Eders Agitationer , og I vil naa Eders onstelige Hensigt , men ikke ' paa nogen anden Maade . " Dirigenten bemærkede , at En af Deputationen fra Matloch visselig havde forbauset dem alle , men han vilde for Nærværende afholde sig fra videre Bemærkninger og opfordre den næste Taler til at troede frem . Denne begyndte : „ Hr. Dirigent ! Mine <b>Damer</b> og Herrer ! Igaar arbeidede omkring tredive tusinde Kirker , Kapeller og Ssndagsstoler alvorligt og med Bon paa at blive Redskaber til at omvende Folket — at lede de Unge paa Dydens Stier , modvirke Syndens Følger ved at <b>vise</b> Syndere til det Guds Lam , og ved det prædikede Evangeliums Kraft standse Ugudelighedens og Elendighedens Strsm . Igaar aabnede Englands Regjering , ved Lov og Fuldmagt , et hundrede tusinde Ugudelighedens Huler, Tilholdssteder for enhver Slags Synd , Ade- og Elendighed , for at modvirke alle kristelige Bestræbelser af Præster ogLcrrere ; derved har den gjort mere for at odelcrgge Legeme og Sjæl , end alle Kirker og Kapeller kan gjore for at frelfe dem . Den mindst fornodne Handel , og den som er en frygtelig Aarfag til Forbrydelser , Fattigdom og Død , sinker alt Godt , udbreder alt Ondt og demoraliserer alle Klasser , er den eneste Handel , som Regjeringen opmuntrer og tillader om Sondagen . Ethvert af disse hundrede tusinde Drikkehuse existerer paa Nationens moralske Bekostning , og intet Menneske burde til eget Gavn eller Fordel fornedre eller demoralisere sin Næste . Vi onsker at faa ophcevet

<h4>1392</h4>„ Ja , dette tror jeg virkelig " , svarede han . „ Det var uden Tvivl en meget sorgelig Begivenhed; men vi har ikke Lov til at domme . Vor Frelser sagde , da Han ved en lignende Anledning talte tillsderne : „ Tro I , at hine atten , paa hvem Siloams Taarn styrtede ned og slog dem ihjel , vare Syndere fremfor alle Mænd , der boede i Jerusalem ? Jeg siger Eder : Nei . " Vi kan fylde vor Uretfærdigheds og Ondstabs <b>Maal</b> og ikke have nogen Plads tilbage for Bodfærdighed og Anger ; thi Guds <b>Aand</b> vil ikke altid virke paa Mennesket ; dog det er ikke godt at domme . „ Hvormange blev ialt reddede ? " spurgte jeg . „ <b>Kvinderne</b> omkom allesammen . Af to <b>Damer</b> — og disse to var de eneste , jeg saa — døde den ene af Kulde , medens hun klyngede sig til Tougvoerket, og den anden faldt overbord og blev af Bolgerue kastet iland . Jeg saa dem begge ligge paa Stranden stive og døde . Af ni hundrede og fem og tredive Sjcrle ombord omkom sex hundrede og forti . " Om Aftenen afholdt de til Modet fra Genf og Brilsfel affedte Gudstjeneste paa Bakken af Skibet for alle de af fine Landsmænd og Landsmandinder, der vilde overvære samme , til aabenbar Mrgrelse for alle Dandsere og Kortspillere . Denne Dag foregik ogfaa en anden Scene ombord, hvortil man ikke kunde være Vidne uden med blandede Folelser . De franske Born — baade Gutter og <b>Piger</b> — samlede sig i en Kreds paa Skandsen og begyndte at synge ikke alene sine egne Nationalsange , men oysaa Sange , der skildrede den sidste forfoerdelige Kng med Tydstland og deres Lidelser under Belejringen , og Bsrnenes Forcrldre

<h4>1788</h4>Man skulde tro , at der ikke kunde være Spaakvinder i vor oplyste Tid , men enhver Slcegt har sikkert nok sine Tosser ; det vilde jo ellers være umuligt for saadllnne Bedragere at faa deres Haandflade nogensinde krydset hverken med Sølv eller Kobber ; dog Tilbøieligheden til at granske Fremtiden er stærk , Vnsket out at forudse tilkommende Begivenheder og kjende noget til Morgendagen er ikke indskrænket til nogen scerstilt Klasse i Samfundet . Men det er en <b>vis</b> Forordning af Forsynet , at det ikke er tilladt os at kjende til noget , som vilde gjøre Livet tungt , ja næsten uudholdeligt . Det Slsr , som er draget over Fremtiden , er draget af Naadens Haand , og den fandeste <b>Visdom</b> er at gribe denne Haand — da kunne vi sikkert vandre vor Vei frem , og vor Fod stal ikke glide . Men vi fe altid , at de Folk , fom ere utaalmodige for Fremtideu og fom gjore Svsrgsmaal og tro paa Forudsigelser , paa Troldmænd , Spaakvinder, Dødningemanere , Tegnsudlæggere , Sortekunstnere og Besvcergere , ere meget taabelige og indskrænkede i sin Erkjendelse , om ikke rent ud slette . Jeg husker en overordentlig stormfuld Aften , Vinden hylede og Regnen vidskede mod Ruden , da en <b>Kvinde</b> kom ind ad Bagdsren . Vi fik hende da ind i Kjskkenet , hvor hun udtalte et Ønske om at tale alene med mig . Da jeg spurgte delt stakkels gjennemblsdede Skabning om hendes Ærinde , stak hun mig et Stykke skiddent Papir i Haanden ; jeg aabnede det og fandt , at det var en trykt Trylleformular , fom kun indeholdt følgende Ord : „ Dn maa putte mig ind mellem Foret paa dit Underliv og faalænge , fom du beholder mig

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012052204016'target=_>Ahlfeld, Friedrich, 1869, Menneskelivet i Guds Ords Lys</a><h4>487</h4>Drenge og <b>Piger</b> deriblandt , som intet Hjerte have for sine Lcerere , som ikte lcenger barnligt hengive sig til dem , men i det Hoieste indtage en klog og beregnet Stilling ligeoverfor dem . I deres Ncervcerelse <b>vise</b> de dem vel en nootorftig Lydighed ; de have vel endnu nogen Frygt i sig for Lovens Bogstav ; men der er ingen hjertelig Lydighed . Den barnlige Wrefrygt , som Born skylde sin Lcerer , fattes hos lun altfor Mange . As Wrefrygten og Idmygheden bor der opstaa en übetinget Lydighed hos Barnet . Lammene folge Hyrden . Ogsaa her gjcelder , hvad strevet staaer : „ I Born , adlyder Eders Forceldre i Herren ! thi dette er ret . " Lcereren udover nemlig en Deel af Foreldrenes Embede . Og for at lcegge alle Born dette ret kraftigt paa Hjerte , siges der i Brevet til Ebrceerne : „ Adlyder Eders Veiledere , og vcerer dem lydige — thi de vaage over Eders Sjele fom de , der stulle gjore Regnstab — at de kunne gjore det med Glcede og ikke snikende ! thi dette er Eder ikke gavnligt . " Enhver Ulydighed mod Lcereren bor Forceldrene betragte som begaaet mod dem selv . En Folge as at der hos Bornene nuomstunder ofte er Mangel paa hine to Grundbetingelser er , at det ogfaa mangler dem van de forste barnlige Dyder i Forholdet til Lcereren , nemlig Kjcerlighed og Taknemmelighed. Intet Mennefie kan stjcenke os noget Storre end den stjcenker os , som gjor os fortrolige med Herren , som med Kjcerlighed , Alvor og Bon vellever os paa Frelsens Vei , som fortscetter vor <b>Uddannelse</b> til Lydighed med vor Herre Jesus Christus , som forer os ncermere hen til Ham , der er Hovedet , nemlig Christus . Ifcmdhed , et faadant Arbeide , et faadcmt Hjerte- og Sjelearbeide er al Tak vcerdt . Med en Lcerer er man ikke fcerdig , nåar man har betalt ham Skatevengene. Man kan ikke betale en Lcerer som en Haandvcerker . For Hjertearbeide tilkommer ham en Hjertetak . Paa en rorende Maade have Hedningerne ofte viist sig taknemlige mod sine Lcerere . Den romerste Keiser Markus Aurelius overoste ei alene sine Lcerere med Rigdomme og Vcerdigheder : ester deres Dod fatte han , Hedningen , endog deres Villeder blandt sine Huusguder . Dette var just ikke saa urigtigt gjort af ham , fom en Hedning ; thi i Sammenligning med de andre Guder skyldte han ialfald sine Lcerere langt Mere end disse . Den gamle romerste Viismcmd Seneka ' s Disciple bragte ham en Dag Gåver . Nu var der En iblandt dem , som var saa fattig , at han ikke kunde bringe ham Noget . Denne traadte da grcedende frem for sin Lcerer og sagdc : „ leg eier Intet ; men jeg bringer dig mig selv , mit Hjerte og min Kjcerlighed . " Og nu christne Born , christne Disciple ! De have jo dog ialfald annammet Mere af gudfrygtige Lcerere end hine Hedninger af

<h4>898</h4>saa sog at erholde en gudfrygtig Mester eller Herre . Den Luft , du hele Dagen bevceger dig i , er af stor Indflydelse Paa dm Helbred . Hvorsomhelst du kommer hen , saa see til at finde dig et Par gudfrygtige Omgangsvenner , med hvilke on kan samtale om Guds Ord og din Frelse . Hoor langt du end kommer bort fra dit Fcedrenehjem , saa skriv flittigt hjem . Ethvert Vrev er en Samtale med dine Forceldre . I ethvert Brev maa du paa en <b>vis</b> Maade trcede frem for dem . Derfor maa du ogsaa gaae paa Raad med dig selv om hvorledes du stal trcede frem for dem . Men for Alt hvad du gjor , vogt dig for al Logn i dine Breve ! Et Logndrev brmger dig mindst hnndred Mile lcenger bort fra dine Forceldre , uden at du behøver at gaae ud af Byen striv ret flittigt hjem . Glem ikke det fjerde Bnd , fordi om du er paa et fremmed Sted . Din Frelser tog det med sia uge op paa Korset ; tag derfor ogsaa du det med dia til det stdste Sted , du scetter din Fod paa . Gjor dig ingen Bekymring for den dermed forbundne Udgift ; alle Forceldre betale gjerne nogle Skillinger for et Brev fra sin Son Har du imidlertid en syg Fader eller en gammel <b>Moder</b> lom har vansteligt for at tilveiebringe Brevpengene , saa betal dem selv . Snlt heller for en Gangs Skyld . Naar Postbuoet bringer din <b>Moder</b> Brevet , feer hun allerede af Frimcerket , at du endnu er en tro og kjcerlig Son . Arbeid derhos ufortrodent . Flyv Me fra den ene By eller Mester Ul den anden . Der groer ikke Grces paa den Steen , som ofte vceltes . En Svend , som opslider sin Fortjeneste i Stovlesaaler, lcerer Intet . Vcer ydmyg ; thi lun Ydmygheden lcerer Roget . Hoffcerdigheden kan Alt og dog tilsidst — Intet Tcenk endelig stedse paa at det egentlige <b>Maal</b> for din Vandring er den salige Evighed , og hvorledes du stal komme tnd , det lcerer Ingen dig saa sikkert som dit Nytestament . Llgesom vi , mine elstede Lcesere have vcrret med den unge Mand paa hans Reiser og Vcmdringer , saaledes ville v : ogsaa vcere tilstede , nåar han ester flere Aars Forlob lommer tilbage til sine Forceldres Huus . Det er en vigtia Dag Hvor opmcerksomt betragte Me hans Fader og <b>Moder</b> ham ! Hvor utaalmodigt vente de Me paa at han riqtiqt Ml oplade sit Hjerte for dem ! De ville vide , hvad han er vleven til . Og hvad erfare de da ? Der er fire Slags Erfarmger, de ifaafald kunne gjore . Een Deel af de unge Mcend er vandret ud fra Hjemmet uden at tage Noget med Ng . Berden og dens Lyst eiede deres Hjerte , da de droq bort , og de eie det fremdeles , nåar de komme tilbage . - Deres Hjerte er kun blevet celore , haardere og fortroligere med kynden . Da staae Frcenderne sorgmodige hos den sjemkomme.

<h4>946</h4>Herre , styr og led vore unge Mcend og <b>Piger</b> saaledes , at de i Valget af sine Livsledsagere ikte bespsrge sig med Kjod og Blod . Lad dem ovlofte sine Dine til Dig og svsrge Dig , om denne Brud eller Brudgom ogsaa ril hjcelpe dem at holde fast ved sin <b>Sjels</b> Brndgom , om han eller hun ogsaa vil lette dem Fremstridningen paa Frelsens Vei . Og svarer Du dem da i den Hellig-Acmd Nei , saa lad det ogsaa vcere Nei , saa lad dem ydmnge sin <b>Sjel</b> under dit Udsagn , som det hoieste og sidste Ord . Ak , Herre , bevar dem , saa Me Skjonhed , Gods og Guld , Stand og Herkomst slutte disse Forbindelser , men den faste Overbeviisning , at t » e begge have een Herre og een Tro , den faste Overbeviisning , at nettop de ville hjcelpe og styrke hinanden paa Veien til det Jerusalem heroventil . Herre , vcer du overalt Trediemand i Pagten ; vcer Du dens <b>Sjel</b> ; vcer Du deres sande Baand ; vcer Du deres Freds og Glcedes Klippe ; vcer Du det <b>Maal</b> , hvortil de begge vandre . Og byg Dig saa Huse , hvori Du selv kan boe , som ere Guds Pauluner hos Menneskene , som

<h4>965</h4>Oromdserne ; men disse gjore sig dog senere gjceldende , og Misforholdet virker da forstyrrende gjennem hele Livet . Der bor hellerikke ved din Fotlovelse opftaae nogen Kloft mellem oig , dine Forceldre og Sostende . leg moa derhos lcegge dig poa Hjertet , at man for Forlovelsen bor have lcert hinanden grundigt at kjende . Man pleier at sige , at man ikke bor stutte Venstab med Nogen , fprend man har sviist een eller to Skjcepper Salt med ham . Et ZSgtesteb er noget langt Mere end et Venstab . Fra en Ven kan jeg trcekke mig tilbage , men derimod ikke fra en <b>Hustru</b> eller Mand . Vogt dig desaarsag for en overilet Forlovelse . leg har svurgt mange LEgtevar , hvis Fred - var bortvegen og som slet ikke kunne stemme overeens , hvor Icenge de havde kjendt hinanden , for de blev forlovede , og da har jeg faaet til Svar : „ FireUger , sex Uger , otte Uger . " Og tilsidst kunde de endog sige : „ Vi kjendte hinanden i Orunden stet ikke . " Spurgte jeg videre : „ Talte I nogensinde med hinanden om Eders Troes Grundvold og Haab , — bad I nogensinde med hinanden , — gik I nogensinde til den hellige Nadverd sammen ? " saa lod Svaret : „ Nei " atter „ Nei " og endnu engang „ Nei . " Hvor stal der da tomme nogen Enighed fra ? Forlovelser , der sluttes i Kjodet , as Kjodets Lyst eller verdslig Beregning , dele alt Kjods Lod . Alt Kjod er Grces , og al dets Yndest som Markens Blomst . Ange <b>Piger</b> , som ingen Luft have til det stille huuslige Liv , som elske Balsalen hoiere end Stuen og Kirken , som med D ) ina gaae ud sor at see vaa Hedningernes Dotre , blive aldrig gode Hustruer . Din Bruds kjcereste Steder bor vcrre Herrens Huus og hendes eget Huus . Kun den <b>Pige</b> bor ncere din Brud , som kan sige til dig som Ruth : „ Hvor du gaaer hen , vil jeg gaae , og hvor du forbliver , vil jeg forblive; dit Folk er mit Folk , og din Gud er min Gud . " Kun med hende kan du vcere <b>vis</b> vaa at komme til at leve i Fred og Enighed . At imidlertid en saadan Pagt aldrig bon sluttes nden Forceldrenes Samtykke , at disse endog bor adsvorges for Bruden selv , og at man forst maa have sikret sig deres Velsignelse , er Noget , hvorom ingen Christen kan vcere i Tvivl . Det Bifald , som efter Forlovelsens Indgaaelse afnodes dem , er intet Bifald , den Velsignelse , som da gives , ingen Velsignelse . Du bor agte deres Ord saa hoit , at du ogsaa stille ydmyger dig under deres Nei . Og vil du <b>vise</b> dem denne 2 Ere , saa bor du allerede adfporge dem , nåar den forste Tilboielighed fremsvirer i dit Hjerte , og forinden du endnu har gjort nogetsomhelst utvetydigt Skridt henimod <b>Maalet</b> . Til disse forste og vigtigste Fordringer komme endnu nogle andre . bor ogsaa med Hen-

<h4>1516</h4>ryste dit Kors ' s Frugter ned i deres Skjod . Ja , velsign vort Kors for os . Du naadefulde og salige Korsorager ! Amen . Mine i Herren elstede.Lcesere ! Der er ingen Stand og Stilling , som Mennesket stunder saa meget ester som 2Egtestanden og en egen Familie . En stor Deel as Ungdommens Moie og Arbeide sigter til dette <b>Maal</b> . Der er Intet , som i hoiere Grad bestjceftiger den unge Mands og <b>Piges</b> <b>Sjel</b> end de Forestillinger , de gjore sig om dette nye Affnit af sit Liv . Der er neppe Nogen , som ikte ester bedste Evne har udmalt sig det med de deiligste Farver . Og dog er der ingen Stand , som bcerer en dybere Sorg og Smerte i sit Skjod end nettop 2 Egtestanden . I mange ZEgtestaber vegynder Kummeren allerede kort ester Bryllupsdagen . Solen er neppe stegen op , forend der lcegger sig morke Skyer foran den . Glcedens Taarer paa Bryllupsdagen efterfolges ofte meget jnart af Sorgens Taarer , Perlerne i Krandsen og paa Halsen af Perlerne paa Kinderne . Og ofte er HEgteflabet en Korfets Gang fra Alteret til Graven . Hvo vil maale al den Hjertesorg og tcelle alle de Taarer , som HEgtestanden har fremkaldt ? Lader os forst betragte det Huuskors , som Gud Herren paalcegger 3 Egtefcellerne selv ! Ikte sjelden er det Tilfceldet , at den Ene af dem allerede bliver syg eller begynder at strante nogen Tid ester Brylluppet . Mange Mcend og Koner har jeg hort klage og sige : „ Det var kun en kort Tid , jeg havde min <b>Kone</b> , min Mand frist ; fra det fyrste eller andet Aar af har min ZEgtestand vceret en Lidelsens Skole , en stadig Sygepleie . " Eller Fattigdommen slutter sig allerede tidligt til det unge Par som en uadstillelig Folgesvend . Eller Herrens Haand hviler tungt paa Nornene . Tidlige tåger Han dem bort , eller Sygdom bliver deres Lod , saa at hverken Nornene kunne glcede sig rigtigt i sit unge Liv eller Forceldrene i Nornene . O kjcere Lceser ! gaaer det ogsaa En og Anden af Eder saaledes , vide ogsaa lat tale om et Morke , som ncesten fra Begyndelsen af har ruget over Eders Samliv , og om andre Perler end dem , som Bruden var smykket med paa Bryllupsdagen , saa feer vel til , at I ikke berove Eder selv Korsets Velsignelse ! Al Trcengsel er en haard og snur Skal , hvori Gud har indesiuttet en kostelig og sod Frugt . Al Revselse synes , medens den er ncervcerende , ikke at vcere til Glcede , men til Bedrovelse; men siden giver den igjen dem , som ved den ere ovede , Retfcerdigheds salige Frugt . Jesus Christus , som har gaaet under Korset og udkaaret det til sin Fane , vil under Korset drage ind til Eder . I det nye Testament seer du ret tydeligt, hvorledes Trcengselen maa aabne Doren for Ham .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008051403001'target=_>, 1834, Evangelisk-christelig Psalmebog</a><h4>1785</h4>En kjcerlig <b>Moder</b> gav du mig , En god , en trofast Fader ! Jeg kjender ikke Livets Vei ; Men deres Raad og <b>Lcerdom</b> , ei Paa Veien mig forlader . 2. O Gud ! lad mig betcenke ret , Hvad fra mit fsrste Acmdedrcet leg deres Omhu stylder ; Da rsres jeg saa inderlig , Og Kjcerlighed til dem og dig Min hele <b>Sjel</b> opfylder . 3. Mit rerte Hjerte bryder ud I Bsn til dig , algode Gud ! Den Svage tan du styrte . Giv , jeg dem lyder redelig ! De lcere mig at kjende dig , Og trofast dig at dyrke ! 4. Len du dem selv med Mistundhed , Og din Velsignelse udbret » Til hvad de forelage ; Og lad dem sinde Trsst hos dig , Naar Alderdommen ncermer sig Med strsbelige Dage ! 5. Lad Dydens Glcede , Haab og Roe I deres Hjerter altid boe ! Dn stedse hos dem vcere ! Og , nåar fra mig du kalder dem Til gode Sjetes rette Hjem , Lon dem med evig A3re !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009091503033'target=_>Ahlfeld, Friedrich, 1866, Menneskelivet</a><h4>134</h4>dem , thi de have meest at gjore med de Smaa , medens Faderen som oftest er mere fjernet fra Bornene ved sine Forretninger og sit daglige Arbeide . Dog skal ingen Fader undfkylde sig og mene , at naar Timotheus's Fader ikke underviste sit Barn i Guds Ord , saa behover han det heller ikke . Thi hiin Fader var en Hedning , som endnu ikke havde bsiet sig under den Hellig Aands Tugt , og hvis han havde villet undervise Sonnen i sin Tro , saa havde det kun veret til dennes Fordervelse . Derfor var det en naadig Styrelse af Gud , at denne Mand , hvis Navn vi iovrigt ikke kjende , lod Moderen og Bedstemoderen opdrage Barnet . Men Du er en Christen . Og om ogsaa den forste Undcrviisning nermest maa blive Moderens Sag paa Grund af hendes Stilling til Bornene , saa vedkommer den dog begge Foreldre i Forening , og i mange Tilfcelde kan Fedrenes Kald og Stilling ikke hindre dem i at opfylde denne Pligt . — Foreldrene paahviler det fremfor Alle at arbeide paa at frelse Barnet . Sknlle de , som i legemlig Henseende kalde Barnet deres , ikke ogsaa vere Fader og <b>Moder</b> til dets aandelige Liv ? Vi vide vel , at intet Barn kan fodes paany ved Underviisning , Formaning og god Opdragelse , men alene ved den barmhjertige Guds Naade i den Hellig <b>Aand</b> . Men , som Paulus siger om Corinthierne : « Jeg avlede Eder i Christo Jesu ved Evangelium " , saaledes kunne alle Foreldre bidrage til deres Vorns Opvcrkkelse og Omvendelse . Da ere de , saa vidt Mennesker kunne blive det , i enhver Henseende Foreldre til disse Born . — Fremdeles ligger det saa ner for Foreldrene , som ernere Barnet i legemlig Henseende , at de ogsaa rekke det den aandelige Nering . Mennesket lever jo ikke alene af Brod , men ved hvert Ord , som udgaaer igjennem Guds Mund . Da ere de atter , saa vidt Mennesker kunne blive det , Barnets Ernerere og Forsorgere . Og netop dem paahviler denne Opvekkelse og Ernering . De have engang ladet Barnet bringe til Daaben ; de have villet , at det skulde blive et Guds Barn , de maae da ogsaa sorge for at gjore dette Guds Barn bekjendt med dets himmelske Fader og rekke det det sande Livets Vrod . De beljende derved i Admyghed :  Herre , vi ere kun Pleieforeldre , Du er den rette Fader " . De legge derved Barnet dagligt i hans Arme , de give det til den , hvem det tilhorer . — Og hvorfor bor dette nu fkee saa tidligt ? Naar vi nylig have hort , at selv smaa Born kunne fole sig saa salige ved at omgaaes med deres Gud , mon da Nogen vilde forholde dem denne Salighed ? Men der kommer endnu andre Grunde hertil . Du vil dog opdrage dine Born . Du indrommer ogsaa , at Opdragelsen maa begynde strax med de forste Aar . Men hvortil vil Du opdrage dem ? Al Opdragelfe maa dog have et <b>Maal</b> < At opdrage Nornene er at drage dem op fra Verden , at drage dem op til vor Gud og Frelser . Vistnok kalde mange Foreldre det ogsaa for Opdragelse at drage deres Born rigtigt ned i Verden , i dens Nydelser og Gleder ; men derom ville vi nu ikke tale . Skal Opdragelsen virke hen til , at Barnet kan blive en Mand ester Guds Hjerte , bestaaer Dannelsen fornemmelig i , at Barnet bliver draget ind

<h4>154</h4>nu , da Guds Straffedom truer ham , begynder han at formane dem , nu vil han fore dem tilbage til Lydighed mod deres himmelske og jordiske Fader . Men hans Ord ere som et mat Pust af Aftenvinden ; de lyde mere som Suk cg Klage end som Irettesettelse og Formaning. Det er for sildigt , Sonnerne ere forstokkede , Skorpen om deres Hjerter er bleven haard . De svare ikke engang deres Fader , de fremture i deres Synder . — Da tåger Gud Herren Straffen i sin Haand og handler saaledes med Elis Huus , at det maa ringe for dens Yren , der horer det . Israel bliver slagen af Philisterne ; Pagtens Ark falder i Fjendernes Hender ; Elis Ssnner omkomme i Slaget ; den Gamle styrter ned af Stolen og knekker Halsen , da han faaer dette Budskab ; hans Svigerdatter , hans Son Pinehas ' s <b>Hustru</b> , foder for Tiden af Skrek . Hun kalder den nyfodte Son Icabod , d . e .  Herligheden " — Israels og Elis Huus ' s Herlighed —  er flyet " , og hun doer . — Det er et alvorligt Skrekkeblllede for denne Elegt , forn kjender saa lidt til Tugt og Lydighed . Man foler , hvorledes Guds Sverd farer ned . Og dog , hvad hjelper Skrekken ? Det gavner kun lidet , at vore Knee ryfte et Bieblik , at vore Kinder for en kort Tid stifte Farve . Alt dette er skrevet os til <b>Lcerdom</b> , til Overbeviisning , til Rettelse , til Optugtelse i Retferdighed , at det Guds Menneske maa vorde fuldkomment , dygtiggjort til al god Gjerning. Hvad vi derfor ifolge Gangen af vore Betragtninger have at lere af denne merkelige og alvorlige Fortelling , kan sammenfattes r de Ord : Foreldre stulle tidligt boie deres Boms Stivsind og venne dem til Lydighed . Det er Guds Raad og Villie . Kjere Leser ! Alt hvad der her i de Troendes Menighed og hisset i de Herliggjortes Menighed stal blive stort , maa fsrst blive fmaat . Den , der ikke i <b>aandelig</b> Forstand er bleven fattig , har ingen Deel i Himmeriges Nige . Hvor Egenvillien ikke er afstaaren og det nye Livs Kvist indpodet , der trives ingen Frugt til Guds 3 Ere . LEngstede Sjele , fonderknuste Hjerter , en opoffret Villie vil Gud have . Lydighed er bedre end Offer . Men stal det blive til Noget med en faadan Opoffrelse af Villien , saa maa der begyndes tidligt derpaa , i den forste Varndomsalder , thi Egeuvillien , Egensindigheden vifer sig allerede tidligt . Den er det forste Vidnesbyrd om Hjertets Hovmod . Daarstab er knyttet til den Unges Hjerte , siger Salomon ( Ordspr . / ! 2 , 15 ) . Betragt engang dine Vorns Liv . Saasnart deres Kraft blot en Smule har udviklet sig , ville de ofte det Modsatte af hvad Faderen eller Moderen vil . Medens de endnu ikke engang kunne udtrykke deres Egensindighed i Ord , legge de den for Dagen ved heftige Skrig og Fagter . Den er saa almindelig hos Alle , at man ofte siger for Spog , at Egensindigheden er Menneskets sjette Sands . Og dog er denne Sands slet ikke at spsge med ; thi deu vorer med hver Dag . Vi kjende <b>Modre</b> , som allerede tyranniseres af deres smaa Born . De havde ikke engang ide forste Aar Mod og Kraft til at bekempe Bornenes Egensindighed ; hvad stal det da , blive til i de modnere Aar , naar Mydigheden bliver stor og sterk

<h4>170</h4>havde . Og det er sandt , at de i en <b>vis</b> Henseende ere uskyldige . Faktiske Overtredelser as Guds Bud kunne de i det fsrste Aar af deres Liv neppe begaae . Heller ikke om Synd i Tanker kan der egenlig vcere Tale , saalcengc Tankerne endnu ikke ere komne til Udvikling eller ere rettede mod et bestemt <b>Maal</b> . Vi ville lade Ordet gjcelde , hvis det skal betyde , at de ere uskyldige , fordi Herren har aftvcettet dem i Gjenfsdelsens Bad med fit hellige , dyrebare Blod . Men derfor ere de dog ikke uden Synd . Et lille Barn , som ligger i sin Vugge under sit hvide Lågen , ligner en Mark om Vinteren , naar Sneen bedcekker den . Alt er stille og roligt , der er endnu intet Spor as selvstcendigt Liv . Herren virker endnu i det Skjulte ; men Fjenden virker ogsaa , saa vidt han kan . Og ligesom der om Foraaret mylrer tusinde Planter , gode og slette , frem af den stille , hvide Mark , saaledes ogsaa af dette Barnehjerte . Sammenlign engang den stille , sde Mark med hvad den er , naar fire eller fem Maaneder ere gaaede . Og sammenlign engang Barnet i Vuggen med hvad det er om fire eller fem Aar ; hist kan Sceden staae i tcette Nader ; men den kan ogsaa allerede ncesten vcere kvalt af Ukrudtet , som var skjult i Jorden . Her kunne Troens fsrste , yndige Blomster allerede vcere fremspirede i Barnets Glcede over Herren og i den barnlig troende Bon ; men her kan ogsaa Synden allerede have overgroet hele det unge Ved . Og hvilket Ukrudt breder sig som oftest fsrst ud over denne Guds Mark ? Vi have nylig talt om Egensindigheden , Egenvillien og Ulydigheden ; men ved Siden heraf staaer ofte Lsgnen allerede i Livets forste Aar . Lsgnen var den fsrste Synd , som betraadte den nye , rene Jord . Slangen sfturgte <b>Kvinden</b> i Paradiset :  Er det virkelig saa , at Gud har sagt : I maae ikke spise af alle Havens Trceer ? " I dette Spsrgsmaal ligger den fsrste Lsgn . Og videre siger den :  I skulle visselig ikke doe " . Her gjsr den Gud til en Lsgner ; det er en anden Lsgn . Og vi kjende jo Alle Slangens Lisgn ; vi maae Alle sige med Augustin :  Du Bedrager , Du Bedrager, hvad er der nu blevet af dit  visselig ikke " , see dog , vi maae Alle dse " . Lsgnen vil rimeligviis ogsaa blive den sidste Synd Pan Jorden ; thi indtil den Time , da Guds Dom farer ned forn et Lyn og knuser alt Selvbedrag , ville Tusinde indbilde sig , at de kunne bestaae for Gud med deres Vantro og dsde Gjerninger . Saaledes er Lsgnen nu ogsaa ester Tingenes scedvanlige Gang som oftest den forste og den sidste Synd i det enkelte Menneskes Liv . Antallet af dem , som i Selvbedrag vandre bort herfra , er overordenlig stort ; og kaste vi Blikket midt ind i Livet , saa er jo ncesten Alt omsnoet og gjennemvcevet af den . Derfor er det en stor og hellig Opgave at kcempe en tapper Kamp uden nogensomhelst Vaabenstilstand med denne Fjende . Saa lader os ogsaa idag drage ud med hverandre til denne Kamp .  En Retfcerdig hader Lsgns Ord " , siger vor Text . En Netfcerdig er enhver troende Christen . Vi ere Alle blevne retfcerdiggjorte ved Jesu Christi Bl « d og ved Troen paa ham . Den , der ikke ringeagter denne Naade , den , der ikke har solgt sin salige

<h4>189</h4>Skolen er en Datter af den christelige Kirke , og Lererne ere Kirkens Medhjelpere ved Opdragelsen af dens unge Medlemmer . Men de ere ogsaa Foreldrenes Medhjelpere ; thi de overtage en Deel af det faderlige Embede . Hvad Foreldrene felv ikke kunne udfore , det udfore Lcererne efter Apostelen Peders Ord :  Som Enhver har faaet en Naadegave , saa tjener hverandre dermed som gode Huusholdere over Guds mangeslags Naade " ( 1 Ped . 4 , 10 ) . la , Lcererne have en ganske anden Betydning i den christelige Kirke end i den hedenske Verden og blandt Israels Folk . I det gamle Testamentes Boger er der ncesten slet ikke Tale om Skoler for Ungdommen . Kun visse Propheter samlede fra Samuels Tid Skarer af Inglinge om sig , forn de uddannede til den prophetiske Gjerning . Disse undervistes i Loven , som skulde foreholde Folket dets Synder , og over hvilken de hellige Spaadomme svevede . De kunde vistnok ikke opdrages til Propheter , thi den hellige Epaadomsgave var en fri Gave af den Hellig <b>Aand</b> ; men Aanden valgte sig dog helst saadanne Redskaber , som ved <b>Kundskab</b> til Loven og ved et stille , aandeligt Liv , bortvendt fra Verdens Larm , bedst vare tilgengelige for dens Stemme . Desuden havde Kongerne og de Megtige i Folket Lccrere for deres Born . Kong David lader saaledes sin Son og Arving Salomon undervise af Propheten Nathan . Men i de fleste Familier paahvilede det Foreldrene at undervise Bornene i Loven og indskcerpe dem den , saa at de aldrig glemte den . Saaledes var det ogsaa Tilfeldet i Herrens og Apostlenes Dage . Der fandtes ogsaa dengang visse Skoler for ' Born ; en saadan havde Gamaliel , ved hvis Fodder Paulus havde siddet . Dog var det kun en lille Skare af 3 ) uglinge , der famledes i en saadan Skole , og om <b>Pigernes</b> Underviisning hore vi slet Intet . De jodiske Skoler , der saa ofte omtales i Evangelierne , vare ikke bestemte for Born , men for Vorne ; da Israel nemlig kun havde eet Tempel , blev Loven forelest og udlagt for Folket i disse Skoler rundt omkring i ' Byerne . De fleste Vorns Underviisning , forsaavidt en saadan virkelig fandt Sted , var lagt i Foreldrenes Hender . Hos Hedningerne var det endnu riugere bevendt med Undervisningen . De rige Familier hos Grekerne og Nomerne benyttede en Slave som Lerer ; denne var en kjobt Trel og blev atter solgt , naar man ikke mere behovede ham . I den hoiere hedenske <b>Kundskab</b> underviste Philosopherne , der ofte famlede store Skarer af Disciple om sig og havde deres egne Skoler . I Apostlenes Gjerninger omtales en <b>vis</b> Tyrannus ' s Skole , hvor Paulus predikede , da han opholdt sig i Ephesus . — Den christne Kirke er det forste store Samfund , som har holdt det for sin Pligt at undervise alle sine Born om Frelsens Vei . Den har givet Skolen og Lererne en verdig Stilling . I den ere Lererne deres . Herres og Frelsers Tjenere overfor Ungdommen , og i den ere ogsaa Nornene altid blevne anviste en barnlig , erefrygtsfuld Stilling overfor Lererne og formanede til en hjertelig Taknemmelighed mod dem . Vi ville nu betragte begges Forhold til hinanden og da navnlig forst dette :

<h4>193</h4>en <b>Moder</b> viser det svageste Barn den storste Omhu , saa gjorer I ogsaa det Samme . Lad Kjcerligheden og Straffen folges ad efter Herrens Erempel , og gid han da ogsaa vil skjemte Eder et Vie , fuldt af kjcerlig Sorg , hvis Blik fcester sig som en Hage i Hjertet og ikke igjen slipper den , det har truffet . — Og naar et Barn kun gjor ringe Fremstridt paa Grund af sine daarlige Anlceg , saa seer dog til , at der idetmindste bliver lagt en god Grund for det Vcesenlige, for <b>Kundskaben</b> om Frelsen . Forsommer ikke at bede med og for Bornene . Naar Landmanden har saaet , oplofter han sine i3ine og beder om Negn , at Herren vil vande Sceden . I saae ogsaa hver Dag , baade Sommer og Vinter ; gaacr da ikke bort uden at oplofte Eders Vine til de hellige Skyer ; thi Herren maa vande . Ingen kan jo kalde lesum Herre uden ved den Hellig <b>Aand</b> ; al Tro og al Helliggjorelse er en Naade , og derfor maa der bedes derom . — Beder ogsaa Herren flittigt , at han vil prcege sit Billede i Eder . I ere tilligemed Forceldrene og Slcegtningene Barnets ncermeste Venner . I trcede det imode , naar det gjor de forste Skridt ud af det fcedrene Huus . Eders Liv stal vcere et Forbillede for det . Gid derfor alle Born maatte mode en sand Vekjender og Esterfolger af Jesus Christus i deres Lcerers Person ! Gid et saadant Villede maatte staae indprceget paa ethvert Barnehjertes blsde Vortavler ! Hvilken Velsignelse vilde ikke derved udbredes over den kjcere Ungdom ! Og omvendt , hvilken Skade kan ikke en vantro Lcerer anrette ved sine Ord og sin Vandel ! Han berover deels Bornene den helligste Glcede i Ungdommen, og deels udsaaer han en daarlig Sced i Hjertet , der slaaer dybe Rodder og ofte ikke kan udryddes mere i hele Livet . Derfor stulle christelige fore noie Tilsyn med deres Skoler , og christelige Forceldre stulle agte paa , i hvilke Hcender de ovcrgive Barnet . Forstanderne for en Bondemenighed , som havde en uvcerdig Lcerer , besvcerede sig engang derover for Bvrigheden og udtalte sig saaledes : „ Naar vore Lam scettes ud paa Marken om Foraaret , tåge vi os vel i Agt for at ovcrgive dem til en udygtig og utro Hyrdes Opsyn . Wen vore Born ere vore bedste Lam . Forst have vi pleiet dem hjemme og underviist dem saa godt , vi kunne ; stulle vi da nu overantvorde dem til en udygtig og utro Hyrde ? Vi gjore ikke engang dette med Lammene , og meget mindre ville vi gjore det med Bornene . " la , kjcere Menighed , Forceldrene bor kunne vente af Lcereren , at han forer deres Born , som ere Christi Lam , paa den rette Grcesgang og vogter dem med Christi Hyrdestav . — Og hvad tor nu Lcereren vente af Bornene ? At de virkelig komme som Christi Lam , og at de medbringe A3rbodighed mod ham som mod den , der arbeider i Christi Tjeneste paa deres Sjcele . Det gjcelder om hele den christelige Menighed , at vi stulle . . forekomme hverandre med sErbodighed " ( Norn . 12 , 10 ) , og det gjcelder fortrinligt om de Embeder , som Herren har indstiftet , idet han bestikkede Nogle til Apostle , Nogle til Propheter , Nogle til Evangelister , Nogle til Hyrder og Lcerere ( Eph . 4 , 11 ) . Her er det da forst og fremmest

<h4>252</h4>til en Hoveddrivefjeder for deres Born . De formane dem til ikke at lade sig overgaae af nogen Anden , ja de fremstille det som en Skam , naar dette skeer . De anspore Barnet ved at sige :  See , at Du kan overgaae den eller den " . De rose Vornene ved ethvert Fremstridt , ved ethvert godt Vidnesbyrd , ja de stille det ene Barn ved Siden af det andet , sammenligne dem med hverandre og fremhceve det ene som et Monster for det andet . Der tales om at staae til Skamme ved Examen , og Vornene opmnntres til at gjore sig og deres Forceldre 3 Ere . Det er bestandig LErgjerrigheden , der stal anspore dem ; men netop dette er en falst Drivefjeder . — Og som Forceldrene , saaledes bcere ogsaa Lcererne sig ad . De forste Pladser i Skolen maae ofte tjene som Loftestcenger for Fliden ; de bedste Charakterer maae vcekke og anspore Arbeidslysten . Mange <b>Modre</b> begynde ncesten regelmcessigt med folgende Scetning , naar de omtale deres Barn :  Han har altid havt de bedste Vidnesbyrd i Skolen , han har altid bragt gode Charakterer hjem . " la , man er gaaet saa vidt , at man har maalt dem med det samme <b>Maal</b> ved Confirmationen og bedomt dem efter deres Flid , deres Nummer og Vidnesbyrd i Skolen . —  Men " , sige I ,  stal man da ikke rose et Barn ? Skal man ikke <b>vise</b> det Anerkjcndelse , naar det har arbeidet trofast ? " Kjcere Christne , Roes og A3re stulle i det Hoieste vcere Krydderi paa Maden , men aldrig Maden selv . Vee det Barn , hvis Strcebsomhed beroer paa Andres Roes ! Vee det Barn , som ikke foler en indre Drift i sig selv til at opfylde sine Pligter ! Det bliver et stakkels , elendigt , ulytkeligt Barn .  Men " , sverger Du atter ,  ere da 3 Ere og Roes ikke et godt Middel til at modarbeide Dovenstab , Uorden og deslige ? " Et Middel ere de vel , som ofte kan bruges med Held ; men et godt Middel ere de ikke , der gives i Sandhed bedre Midler . At fore Barnet rigtig ind i Samfundet med Herren , at <b>vise</b> det , at Skolegang og Underviisning ere at betragte som dets guddommelige Kald og som Noget , hvorved det tjener Gud , at bede med det om et frist , flittigt Hjerte , at formane det til bestandigt at bekjende sin Synd , ogsaa sin Dovenstab og Forsommelighed for Herren : det er et bedre Middel . Ved at benytte LEvgjerrigheden som Middel driver man maastee een Djcevel ud , men aabner sikkert Doren for en anden , som senere er meget vanskelig at faae ud , som omtlamrer hele Barnets Hjerte og ofte hele Livet igjennem bliver boende i det som en Fjende af dets sande Vel . Kjcere Lcesere , naar man trcekker en Blomst med dens Rodder op af Jordens Skjul og udscetter dem for Solens Lys , saa visner Blomsten . Vore Livsrodder maae ogsaa hvile skjulte og stille i Gud , og af ham snge sin Ncering , den tilkommende Verdens Krcefter , som stulle bevare det hele Menneske . Men den , der bliver opelstet , dreven og tilskyndet af Lyst til 3 Ere og Roes , har losrevet sine Livsrodder fra Gud og udbredt dem for Luften og Solen . Han soger sin Styrke i Mennesker og i dens Dom . Herren er ikke mere hans Styrke ; han har gjort et ulykkeligt Bytte . Hos et saadant Menneske

<h4>260</h4>og koldt Verelse eller i det Frie , saasnart Aarstiden tillader det . Al tidlig Noes og LEre er Drivhuusvarme , kjcere Venner . Vernene fkyde op i <b>Kundskab</b> , men endnu mere i hsie Meninger om sig selv . Hold dem derfor i Skjul , hvor Offenlighedens Lys ikke treenger stoerkt ind , og stil dem paa et kjsligt Sted , hvor LErens Yppige Varme ikke naaer hen ; thi da faae de en fast Stamme , fsrend de udfolde sig for det offenlige Liv . Der skal sterke Been og brede Skuldre til at boere Roes og 3 Ere ; fremfor Alt maa der Idmyghed til , ellers bliver vor 3 Ere til Skamme . Saaledes have ogsaa altid Guds Veie voeret , saaledes har han opdraget sine Helte og Kemper . David , hans herligste Konge i Israel , vogtede Faar lige til sin Anglingsalder ; de store Gaver vare allerede tilstede , men der var Ingen , der beundrede dem . Om Marias , Herrens Moders Ungdom vide vi slet Intet , ikke engang hvad hendes Fader og <b>Moder</b> hed ; thi de Navne , som Legenden har overleveret os , findes ikke i Skriften . I Stilhed vcrede hun op ; i denne Stilhed udfoldede heudes Mmyghed sig , og formedelst den agtede Gud hende vcerdig til at blive hans eenbaarne Ssns <b>Moder</b> . Om Apostlenes Ungdom vide vi nesten Intet . De fleste af dem have fisket i det galileiske Hav , og i denne Stilhed har Gud ladet de Charakterer modnes , som Herren senere brugte til sin Naadegjerning . — Vil Du nu have , at dine Bsrn engang paa den dem anviste Plads skulle blive dygtige Redstaber i Guds Tjeneste , fromme Borgere i Guds Rige og brave Mcend og <b>Kvinder</b> i deres Foedreland og Huus , saa unddrag dem for den forfengelige Mres giftige Aandepust , der frembringer en hurtig , men sygelig Udvikling , og snart lader dem afblomstre selv i Menneskers Vine , ja tilsidst forderver dem paa Legeme , <b>Sjel</b> og <b>Aand</b> . Hvad der skal trives , blomstre og boere god Frugt , maa have sin Rod i Gud og vore langsomt i ham . Giver han da A3re , saa er denne 3 Ere ikke forfengelig . — Naaderige Gud , saa lad da os og vore Born vore , ikte i den verdslige 3 Eres kunstige Lys og Feberhede , men i din Naades Solskin . Amen .

<h4>364</h4>Liv i den hellige Skrift . Kun om det sande Midtpunkt i Livet , om hendes Forhold til Christus aflegger Fortellingen os Beretning , idet den siger , at hun var en trofast Discipelinde af Herren . Lucas forteller endvidere , at hun var rig paa gode Gjerninger og gav mange Almisser ; og endelig tale Enkerne om hende , da de staae gredende omkring Liget og <b>vise</b> Petrus de Kjortler og Klceder , som Tabitha havde gjort , den Stund hun var hos dem . Vi kunne heraf slutte , at det ikke har manglet hende vaa jordisk Gods og Eiendom . Men vi vende tilbage til hendes Arbeider . Hun havde lert , hvad der var Brug for i Livet , hun havde ikke faaet en forfengelig Flitterdannelse, men en gudfrygtig , praktisk <b>Dannelse</b> ; og dette er et Vink til Eder , I Msdre , saavelsom til Eder , I Dsttre . Den romerske Keiser Augustus lod sine Tottre flittigt spinde Uld , og selv Fyrst - inder fkammede sig ikke i gamle Dage ved at sidde i samme Verelse som deres <b>Piger</b> og spinde . Den unge <b>Pige</b> skulde dengang lere , hvad hun havde Brug for i Livet ; men nu lcere de Unge saa Meget , som de sjeldent eller aldrig faae Brug for , og forsomme undertiden ganske det Nødvendigste . De tilbringe deres Tid med den fine Pynt , med hvad jeg vilde kalde Kniplingerne og Kantuingerne om det Nodvendige, medens de kun tarvelig ! lcere det , der fsrst og fremmest behsves. En cegte <b>Huusmoder</b> stal forst tåge fat paa Kjernen i Huusvesenet, og bliver der da Tid tilovers , kan hun siden beskjeftige sig med de prydende Indfatninger . Den befrugteude Negn strommer ikte ned fra Skyens brogede 9 tand , men fra dens morke , eensfarvede Midte . — Denne Ulyst til et bestemt , anstrengt Arbeide er for en Deel trengt ind i alle Steender . Her hjemme hos os er der mange <b>Piger</b> , som ikke ville tjene og foretrekke allehaande andre Beskjeftigelser for denne Stilling . Navnet <b>Tjenerinde</b> , som Herrens <b>Moder</b> ikte stammede sig ved , er blevet de fleste af dem modbydeligt . Man soger et let , frit Liv med saa rigelig Fortjeneste som muligt . Men troe I , at en ung <b>Pige</b> paa denne Maade uddannes til at blive en god <b>Huusmoder</b> ? Hvorledes kan dette dog stee , naar hun ikke itide lerer en Huusholdning at kjende ? Den unge <b>Pige</b> skal opdrages for Huset , og dette steer derfor bedst i Huset , enten under Moderens Opsigt eller i Tjeneste hos en anden Familie . Mangfoldige ulykkelige A3gteskaber opstaae deraf , at <b>Konen</b> har tilbragt sin Ungdom med allehaaude Sysler , som ere Hnusvesenet uvedkommende . Nu hviler Huusholduingen med sine daglige , eensformige Pligter paa hende som en Centnervegt . Hun har ikke vennet sig til hertil i sine unge Aar og stal nu rogte et Kald , hvis Lereaar hun ikke har gjennemgaaet. — Ville I endelig sikkre Eders Born en velsignet Huusstand, saa opdrager dem ikke for Verden . Hjemmet og Verden , det Snevre og det Vide , passe ikke sammen .  Men hvad stal der blive af Barnet , naar det opvorer i en saadan Stilhed ? " sporge maastee Mange . Det Samme , som der bliver af Blomsten , der opvorer i den stille Dal i Skjul af de store Treer . Morgenduggen svinder ikke bort af dens Blade , men bliver liggende paa dem til hoit oft

<h4>469</h4>Herre , styr og led vore unge Mcend og <b>Piger</b> , at de ved Valget af en Wgtefcelle ikke raadfsre sig med Kjod og Blod , men med Dig . Lad dem oplofte Vinene til Dig og sporge Dig , om denne Brud eller Brudgom kan hjcelpe dem til at bevare deres Troskab mod deres Sjcels Brudgom , til at gaae frem Paa Frelsens Vei . Og hvis Du i den Hellig <b>Aand</b> svarer dem Nei , lad saa deres Sjcel ydmyge sig under dit Udsagn som det hsieste og sidste Ord . O , Herre , bevar dem , at ikke Hensyn til Skjonhcd , Penge , Gods og Familieforhold bliver det Bestemmende for dem , naar de indgaae en Forening , men alene den faste Overbeviisning , at de begge have een Herre og een Tro , den faste Overbeviisning , at de kunne styrke og hjcelpe hinanden paa Veien til det himmelske Jerusalem . Herre , vcer Du overalt den Tredie i Forbundet , vcer Du Sjcelen i det , det sande Baand , der knytter det sammen , vcer Du Fredens og Glcedens Klippe , vcer Du det <b>Maal</b> , hvorefter de begge vandre . Og byg Dig saa Huse , hvori Du kan boe , Huse , som ere Guds Pauluner hos Menneskene ( Aab . 21 , 3 ) , som ere Haller ved din hellige Kirke og Forhaller til de evige Boliger hist oppe . Det skjcrnke Du vor Ungdom for Jesu Christi Skyld i den Hellig <b>Aand</b> . Amen . Gud sagde i Begyndelsen til Manden :  Det er ikke godt , at Mennesket er ene , jeg vil gjore ham en Medhjcelp til at vcere hos ham . " Og det samme siger han endnu . Mangen har han vistnok givet at leve ugift . Blandt Apostlene og Chnsti fsrste Vidner mode vi Mcend som Paulus og Barnabas , der for Herrens Skyld vilde leve ugifte . Men Paulus vcerger sig udtrykkeligt imod den Formodniug, at han skulde ansee den ugifte Stand for udnicerket ved en sceregen Hellighed . Vistnok skriver han til de Ugifte og Enkerne , at det er godt , om de blive , som han er , og han giver lomfruerne det samme Raad , men fsier til , at han raader til dette formedelst den nccrvccrende Nsd . Derfor kan der ogfaa komme ulykkelige Tider i Fcrdrelaudet , i Kirken og i Familierne , hvor man endnu bor agte paa det samme Raad . Men hvis en saadan Afholdenhed giver Djcevelen Leilighed til at friste og besmitte Sjcelene , saa raader ogsaa Paulus til at gifte sig , selv i saadcmue ulykkelige Tider . Han siger :

<h4>487</h4>Brud , saa kan Du vcere <b>vis</b> paa , at der vil herske Enighed imellem Eder . — At nu en saadau Forbindelse ikke kan sluttes uden Foreldrenes Velsignelse , at de fremfor alle Andre maae adspsrges , og at der maa bedes om deres Velsignelse , det burde staae fast i enhver Christens Hjerte . Det Samtykke , som afnodes dem efter Forlovelsen , er ingen Velsignelse . Du bsr agte deres Mening saa hsit , at Du endogsaa bsier Dig for deres Nei , og vil Du vife dem den rette Mresrygt , saa raadfsrer Du Dig allerede med dem , saasnart Tilboieligheden begynder at spire i dit Hjerte , fsrend Du endnu har gjort et eneste bevidst Skridt mod dit <b>Maal</b> . — Til disse fsrste Fordringer slutte sig nu andre . LEgtefolk bsr ogsaa med hensyn til Alderen staae i det rette Forhold til hinanden . Det har sjeldent viist sig vel gjort , naar ynglingen har rakt sin Haand til en meget celdre Brud . — Den <b>Hustru</b> , som Manden vcelger sig , maa kunne forestaae sit Huus godt ; den Mand , som rcekker en <b>Pige</b> sin Haand , maa kunne rsgte et cerligtKald og erncere sin Familie ved sit Arbeide . Det er en ynkelig Mand , eller egenlig ingen Mand , som ssger sig en <b>Hustru</b> for ved hendes Midler at kunne fore et dovent og uvirksomt Liv . Han vinder aldrig sin Hustrues Agtelse . Naar Forelskelsens fsrste Ruus er forbi , siger hun ham lige op i Ansigtet :  leg har gjort Dig til en Mand . " — Manden maa kunne sige om sin <b>Hustru</b> :  Ved Guds Naade har jeg sundet en kostelig Perle " , og Hustruen om Manden :  Gud har skjcenket mig en sikker Støttestav , hvorved jeg trostig vandrer igjennem Taaredalen . " Men vistnok bliver Herren altid den bedste Støttestav for dem Begge . — Hvad vi her have omtalt , er Grundbetingelserne for enhver christelig Forlovelse . Fsier Gud Herren da endnu noget Andet hertil , som Stjonhed , Formue, Duelighed i en eller anden Retning , saa er det en god Tilgift ; men det bsr ikke indtage den forste Plads . Hedningen Themistokles vilde hellere have en Mand til sin Datter , som trcengte til hendes Penge , end Penge , som trcengte til en Mand . O , hvor ofte sinder ikke det Omvendte Sted hos os Christne ! Vogt Dig , at Du ikke forst tomter paa Tilgiften og Rammen om Billedet . Forst Kjernen og saa Skallen ; forst Billedet og saa Rammen ! Ssger Du rigtigt , beder Du rigtigt , saa vil Herren ogsaa ssrge for , at Du sinder . Han bygge dit Huus i Naade . Amen .

<h4>645</h4>kunne forkynde Guds vidunderlige Gjerninger for hele deres Vennekreds. I den Stad , hvori jeg boer , levede for ikke lcenge siden en gammel , blind <b>Kone</b> , som var til Vederkvegelse for mange torstige Sjele . Hun kunde ganske anderledes vidne om det Lys , som har skinnet i Merket , end de fleste Folk med sunde Bine . — I Aaret 1739 sendte Vrodremenigheden to Missionairer til Vestindien . Den ene as dem , Albin Theodor Feders , havde sinderet ved tre Universiteter og var en dygtig Theolog , om hvem man ncerede store Forhaabuinger. Den anden , Christian Gottlieb Israels , var Skreddersvend og halt og pukkelrygget . Udvortes saae det staktels Menneske ynkelig ud ; men indvortes stod det ypperligt til med ham ; der var han saa lige og rank som et Grantre , strakte sig i fast Tro og barnlig Kjcerlighed op mod sin Herre og vilde kun tjene ham ved at udbrede hans Rige blandt de stakkels Negere . At han selv formaaede noget , havde han ikke den fjerneste Tanke om ; han kjendte sin Svaghed , Herren skulde <b>vise</b> sin Kraft i ham . Deres <b>Maal</b> var Ben St . Thomas . Indtil St . Eustatius reiste de med et dansk Skib . Derfra fortsatte de Reisen med et engelsk Fartsi . Dette led Stibbrud paa en lille Klippes . Den storste Deel af Mandskabet reddede sig i Land i Skibsbaaden ; men de to Missionaircr maatte blive i Skibet . Forend dette git under , fandt de dog et Tilflugtssted paa en lille Klippe . Feders forssgte at naae Land paa en Rekke Stene , som ragede op af Havet , men gled ud og fandt sin Dsd i Velgerne . Den halte Israels blev tilsidst trukken op paa Z3en ved Hjelp af et udkastet Toug og saaledes reddet . , Da tenkte Mangen :  Den gode Arbeider ligger i det dybe Hav , og Krobliugen lever " . Og dog blev denne Krobling med sit troende , salige Barnehjerte og sit freidige Vidnesbyrd til stor Velsignelse for de Sorte paa St . Croix og St . Thomas . Det Ringeagtede udvalgte Gud . Aagre kun med dit Pund . Ved kun om Troskab ; Velsignelsen er Herrens Sag . Lyset skinner desto klarere ud af det stakkels , skrsbelige Kar . Her seer man ret , at det er Herren , der gjor det . Ham tilhorer LEren , og han udretter ofte sin Gjerning ved de Svage . Og selv om et Barn er saa svagt paa Legeme og <b>Sjel</b> , at man slet ikke sporer nogen Virkning af Guds Ord paa det , og at det slet Imet kan gjsre til Guds A3re , saa stal man dog ikke fortvile. Jeg har seet Foreldre grede de bittreste Taarer , fordi deres Barn hverken kunde stedes til Consirmation eller til Nadverens Nydelse . Men derfor er jo dog Hunlen ikke lukket for det . Barnet er dsbt . Gud har antaget det som sit Barn og sin Arving , og det har jo ikke ved Gjenstridighed forspildt denne Arvelod , men Guds megtige , übegribelige Haand hviler paa det . Knnue vi ikke nenne os til denne <b>Sjel</b> med Ordet og den hellige Nadvcre , saa er det jo derfor ikke sagt , at Herren ikke kjender nogen Vei til den . Bliver kun ved at haabe og bede . Der vil komme en Dag , da Sloret bliver borttaget fra disse Ulykkelige , da Naaden ogsaa skinner i disse tillukkede Kar , da disse Statler stulle glede sig med uudsigelig Glede ,

<h4>772</h4>Lidelse og Dsd . Du har forvandlet Korset fra en vanvcerende Marterpel til et Livsens Troe , hvis Blade tjene til Folkenes Legedom ( Aab . 22 , 2 ) . Ja , Du har helliget Korset . Ogsaa vort Kors skal blive en naaderig Vyrde til Frelse for vore Sjele . Naar det tynger os , skulle vi lere at finde Dig , slutte os ncermere og inderligere til Dig , afdse fra os selv , nedsenke os i Dig og blive Eet med Dig . Ved ethvert Kors stal Verden mere og mere korsfestes for os . — O Herre Jesus , saa velsign da alle vore Trengsler og Sorger , de smaa med de store . Ler os at forjage al Knurren og Utaalmodighed . Lad os have den salige <b>Bevidsthed</b> , at Du viser os Tiltro ved at gjere os til dine Lidelsesbrsdre . Og som den himmelske forlosende og frelsende Kjerlighed ikke forlod Dig under Korsets Byrde , saaledes bevare Du den ogsaa levende i os . Ja , lad dens Lys straale midt i Sorgens Nat ; lad os takke Dig , fordi Du tugter os ; lad os , selv naar Korset tynger paa vore Skuldre , rekke vore Vrsdre en tjenende Haand , at vi netoft i Lidelsens Dage maae kunne hjelpe til at ryste Frugterne as dit Kors ned i deres Skjod . Ja , Du naaderige og salige Korsdrager , velsign vort Kors . Amen . Kjcere Lesere ! Der er ingen Stand og Stilling , som Mennesket lenges saa meget ester , som Mgtestanden og sin egen Huusstand . En stor Deel af Ungdomsaarenes Arbeide og Besver styrer hen mod dette <b>Maal</b> . Der er Intet , hvormed Anglingens og den unge <b>Piges</b> <b>Sjel</b> hellere beskjeftiger sig end med de Billeder , de danne sig af dette nye Afsnit i deres Liv . Der er vel Ingen , uden han jo , saa godt han formaaede , har udmalet sig det med de skjønneste Farver . — Og dog er der ingen Stand , der kan vere saa rig vaa dyb Hjertesorg og Smerte som netov AZgtestanden . I mange Mgtestaber begynder Kummeren snart ester Bryllupsdagen . Neppe er Solen staaet op , forend msrke Skyer skjule den . Gledestaarerne paa Bryllupsdagen efterfslges ofte meget hurtigt af Sorgeus Taarer , Perlerne om Halsen af dem paa Kinderne . Ja , ofte er AEgtestanden en Korsets Vei fra Alteret til Graven . Hvem kan vel telle alle de Taarer , som ere udgydte i Mgtestanden ? Vi ville fsrst begynde med de Kors , som Gud Herren selv paalegger 3 Egtefolkene . Ofte bliver en af dem angreben af en Sygdom eller Svaghed kort efter deres Forening . Jeg har hsrt mange Mend og Hustruer klage over , at deres LEgtefelle kun en kort Tid havde veret rask , at deres 3 Egteskab havde veret en Lidelsesskole lige fra dets Vegyndelse , en bestandig Sygepleie . Eller Mangel og Savn slutte sig tidligt til det unge Par som en uadskillelig Fslgesvend. Eller Herrens Haand hviler tungt paa Bsrnene . Han tåger dem tidligt bort , eller de hjemssges af en eller anden Svagelighed , som volder Foreldrene Sorg og forbittrer Vsrnene Gleden over deres unge Liv . — O , tjere Lesere , hvis det gaaer en af Eder saaledes , hvis I kjende de nwrke Skyer , der hurtigt have fordunklet Bryllupsdagens Sol , hvis I kjende hine Perler , der have aflsst dem i Vrudepynten , saa vaager over , at I ikke forspilde Eder

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092403020'target=_>Ahlfeld, Friedrich, 1865, Menneskelivet i Guds Ords Lys</a><h4>265</h4>Dig ! Hjelp os , faa vi aldrig mere kunne have Andet i Tanke end din og aldrig mere kunne tåge Andet i vor Mund end hvadsomhelst der er fandt , hvad der er erbart , hvad der er retferdigt , hvad der er reent , hvad der er elsteligt, hvad der tales vel om ! Begynt » at velsigne os med den Naade , som de Salige besidde , der ikte mere knnne losrive sig fra Guds Villie og Sandhed , der efter at vere fuutNe til Hans Hjerte ogfaa staae faa faste i Hans Veie , at de ilke mere vide eller ville noget Andet end Ham og hvad der er ret for Ham . Giv os ogfaa en fant » Gru for Lognen , der , fnin fodt af Djevelen , aldrig kan fore Andet end Forbannelse med sig . Giv os og vore Born alt dette af Naade i den Hellig-Aand ! Amen . Mine i Herren elskede Lesere ! Man taler saa gjerne om nstyldige Born . Man fremstiller ofte de Småne , som om deres Hjerter vare aldeles pletfrie . Det er fandt : icn <b>vis</b> Forstand ere de uskyldige . I Gjerningen kunne de i det forste Aar af sit Liv vanskelig overtrene Guds Bud . Selv om Syud i Tanter kan der ikle vere synderlig Tale , saalenge Tankerne endnu ikke ere komne til Udvikling og endnu ilke ere rettede paa noget klart <b>Formaal</b> . Vi ville lade hiint Munnhelt » gjelde , nåar man tager det i den Forstand , at de ere uden Skyld , fordi Herren i Gjenfodelsens Bad hnr aftoet dem med sit hellige , kostbare Blod . Men derfor ere de dog ikke udeu Synd . Et lidet Varn , som ligger i sin Vugge under sit hvide Teppe er som en fnebedceklet Mark . Alt er glett og jevnt ; der er endnu intet Spor til selvstendig! Liv nt see . Herren arbeider endnu i Stilhed derinde ; men ogsaa Fienden arbeider der , saavidt han kan . Og ligesom Planter i Tnsindviis , gode og onde , bryde frem nf den stille , hvide Mark , nåar Vaaren kommer , faaledes ogsnn nf dette Barnehjerte . Sammenlign engang den stille , ode Mark med hvad den er en sire , fem Macmeder fenere ! Sammenlign Barnet i Vuggeu med hvnd det er en sire , fem Aar senere ! Pnn Marken kan da Hveoen stane Strna vet » Strnn ; men det i Jorden skjulte Ukrud knn ogsaa nesten have kvalt den . Hos Barnet tan Troen og Tilliden til Herren allerede have udfoldet sine forste deilige Blomster i barnlig Glede og eenfoldig Von ; men Synden knn ogsnn allerede have overgroet hele det unge Bed . Og hvilket Ukrud pleier forst at ud brede sig over denue Guds Mark ? Vi have nylig omtalt Egensindigheten , Egenvillien og Ulydigheten . Men ved Siden deraf staaer i Livets Morgen ogsnn ofte Lognen . Lognen var den forste Synd , som betrnadte den nye , rene Jord . Slcmgeu sporger <b>Kvinden</b> i Paradiis : „ Har Gud virkelig sagt : „ „ I stulle ikke ede af alle Treer i Haven " " ? I

<h4>296</h4>Drenge og <b>Piger</b> teriblcmdt , forn intet Hjerte have for sine Lerere , som ikke lenger barnligt hengive sig til dem , men i det Hoieste indtage en klog og beregnet Stilling ligeoverfor dem . I deres Nerverelse <b>vise</b> de dem vel en nodtorftig Lydighet»; de have vel endnu nogen Frygt i sig for Lovens Bogstllv ; men der er ingen hjertelig Lydighet » . Den barnlige 3 - Erefrygt , forn Born stylte sin Lerer , fattes hos kun altfor Mange . — Af og Idmygheden bor der opstnce en übetinget Lydighet » hos Barnet . — Lammene folge Hyrden . Ogsaa her gjelder , hvad strevet staaer : „ I Born , adlyder Eders Foreldre i Herren ! thi dette er ret . " Lereren udover nemlig en Deel af Foreldrenes Embete . Og for at legge alle Born dette ret kraftigt paa Hjerte , siges der i Brevet til Ebreerne : „ Adlyder Eders Veiledere , og verer dem lydige > — thi de vaage over Eders Sjele som de , der stulle gjore Negustab — at de kunne gjore det med Glede og ikke sukkende ! thi dette er Eder ikke gcwnligt . " Enhver Ulydighed mod Lereren bor Foreldrene betragte som begnnet mod dem selv . En Folge nf nt der hos Bornene nuomstunder ofte er Mangel paa hine to Grundbetingelfer , er nt det ogsnn mangler dem paa de forste barnlige Dyder i Forholdet til Lereren , nemlig Kjerlighed og Tnknemmelighed. Intet Menneske kan stjenke os noget Storre end den stjenker os , som gjor os fortrolige med Herren , som med Kjerlighed , Alvor og Bon veileder os pna Frelsens Vei , som fortsetter vor <b>Uddannelse</b> til Lighed med vor Herre Jesus Christus , som forer os nermere hen til Hnm , der er Hovedel , nemlig Christus . Iscmdhed , saadcmt Arbeide , saadcmt Hjerte- og Sjelenrbeide er al Tak verdt . Med en Lerer er man ikke ferdig , nåar man har betalt ham Skolepengene . Man kan ikke betale en Lerer forn en Hnnndverker . For Hjertearbeide tilkommer hnm en Hjertetnk . Paa en rorende Maade have Hedningerne ofte viist sig tnknemmelige mod sine Lerere . Den romerske Keiser Marcus Aurelius overoste ei aleue sine Lerere med Rigdomme og Verdigheter ; efter deres Dod fatte han . Hedningen , endog deres Billeder blandt sine Huusguter . Dette var just ikke saa urigtigt gjort af ham , forn en Hedning ; thi i Sammenligning med de andre Guder styldte han inlfnld sine Lerere langt Mere end disse . Den gamle romerske Viismcmd Senecn ^ s Disciple bragte ham en Dag Gaver . Nn var der En iblandt dem , som var saa fattig , at han ikke kunde bringe ham Noget . Denne traadte gredende frem for sin Lerer og sagde : „ leg eier Intet ; men jeg bringer dig mig selv , mit Hjerte og min Kjerlighed . " Og nu , christne Born , christne Disciple ! De have jo dog ialfald annammet Mere af gudfrygtige Lerere eud hine Hedninger

<h4>588</h4>Tag . De vide endnu ikke , med hvilken Skade for sin <b>Sjel</b> og for et heelt langt Liv de ofte maae betale en faadcm Frihed . De forstace endnu ikke , at de ved at arbeide iet eneste Fag ingen <b>Uddannelse</b> saae for hele Livet . Det huuslige Liv maa nodvendigviis gjore en Kvinte mere stikket for hendes tilkommende Huus end Livet i en Fabrik . Hvad kan en <b>Kvinde</b> forrette i sit Huus , der som <b>Pige</b> har forferdiget Pynt , Blomster , Kroller , Cigarer , der som <b>Pige</b> har stanet ved en Hurtigpresse og vanlagt og aftaget Arkene eller passet en Spindemastine eller i et Sukkerrafsinaderi gjennem hele Macmeter drevet et eensidigt og eensformigt Arbeide ? I troe , at saadcmt Arbeide lonner sig bedre ; men det er ikke Tilfeldet . I gaae tabt af den stille huuslige Glede og af den paa lenge tilvant Arbeide grundede Uddcmnelfe til en brav og dygtig <b>Huusmoder</b> ; ak , I tale ofte endnu langt Mere ! Naar Eders eget Huus siden falder Eder for stille , nåar I hverken have Sands for eller Ovelse i dets mcmgehacmde smaae Sysler , nåar I ikke ere Bestyrelsen af en simpel Huusholdning voxne , nåar den egtestabelige Fred strander paa disse Skjer , da ville I engang indsee , til hvor stor Skade den tidlige Selvstendighet , og rigeligere Fortjeneste har veret Eder . Fordi Josef havde tjent , kunde han siden herske ; fordi han havde lert at forvalte en Andens Gods , kunde han siden forvalte sit eget . Men det Kosteligste, han havde bragt med sig fra sin Tjeneste hos Potifar og i Fengslet , var Idmygheden . Og legger nu rigtigt Merke til vort Brev ! Onesimus har veret Trel hos Filemon, en Christen i Kolosse i Asien . Han har lotet bort fra ham . Hvorledes han maastee har flakket om , hvor meget Ondt han maastee har doet , derom siger Skriften ikke et Ord . Om det har gacet ham vel eller ilde , derom staaer der Intet strevet . Men rimeligviis har han lidt Ondt . En saadan bortromt Trel betragtedes jo hos Romerne som et af Buret undfluppet vildt Dyr . Jegerne med alle sine Hunde vare i Helene paa ham . Onesimus flygler til Rom . I den store Vrimmel af to Millioner Mennesker kan en Flygtning lettest skjule sig . Vistnok gaaer ogsaa der hans <b>Sjel</b> lettest tabt , om ellers Herren endnn har nogen Deel i den . Og Herren havde endnu Deel i Onesimus ' s <b>Sjel</b> ; ja . Hau vilde have den heel og holden . Han lod ham paa sin vildfarende Vei komme til den fengslede Paulus . Og Paulus bringer ham ved sin veldige Prediken , veldig selv i hans Lenker , til den Herre lesum , der alene formaner nt frigjore Mennesket . Kun den , som Sonnen faner frigjort , er virkeligen fri . Onesimus erkjendte , nt den Frihed , hnn ved Svig havde opnaaet , var en falst Frihed . Han lod sig

<h4>670</h4>Herre , styr og led vore unge Mend og <b>Piger</b> saaledes , at de i Valget af sine Livsledsagere ikke besporge sig med Kjod og Blod . Lad dem oplofte sine Oine til Dig og sporge Dig , om denne Brud eller Brudgom ogsaa vil hjelpe dem at holde fast ved sin <b>Sjels</b> Brudgom , om han eller hun ogsaa vil lette dem Fremstridniugen paa Frelsens Vei . Og svarer Du dem da i den Hellig-Acmd Nei , saa lad det ogsaa vere Nei , saa lad dem ydmyge sin <b>Sjel</b> nnder dit Udsagn , som det holeste og sidste Ord . Ak , Herre , bevar dem , saa ikke Skjonhed , Gods og Guld , Stand og Herkomst stutte disse Forbindelser , men den faste Overbeviisning , at de begge have een Herre og een Tro , — den faste Overbeviisning , at uettop de ville hjelpe og styrke hincmden paa Veien til det Jerusalem heroventil . Herre , ver du overalt Trediemcmd i Pagten ; ver Du dens <b>Sjel</b> ; ver Du deres sande Baand ; ver Du deres Freds og Gledes Klippe ; ver Du det <b>Maal</b> , hvortil de begge vandre . Og byg Dig saa Huse , hvori Du selv kan boe , som ere Gnds Pauluner hos Menneskene , som

<h4>690</h4>vel for Oieblikket Grendferne ; men de gjore sig dog senere gjeldende , og Misforholdet virker da forstyrrende gjennem hele Livet . Der bor hellerikke ved din Forlovelse opstace nogen Kloft mellem dig , dine Foreldre og Sostende. Jeg maa ogsaa legge dig paa Hjertet , at man for Forlovelsen bor have lert hincmden grundigt at kjende . Man pleier at sige , at man ikke bor stutte Venstab med Nogen , forend man har spiist een eller to Skjepper Salt med ham . Et er noget langt Mere end et Venstab. En Ven kan jeg trekke mig tilbage fra , men derimot » ikke fra en <b>Hustru</b> eller Maud . Vogt dig desaarfag for en overilet Forlovelse . Jeg har fvurgt mange 2 Egtepar , hvis Fred var bortvegen og som slet ikke kunde stemme overeeus , hvor lenge de havde kjendt hincmden , for de bleve forlovede , og da har jeg facet til Svar : „ Fire Uger , sex Uger , otte Uger . " Og tilsidst kunde de endog sige : „ Vi kjendte hincmden i Grunden slet ikke . " Spurgte jeg videre : „ Tnlte I nogensinde med hinnnden om Eders Troes Grundvold og Hnnb , — bad I nogensinde med hincmden , — gik I nogensinde til den hellige Nadverd sammen ? " saa lod Svaret : „ Nei " og atter „ Nei " og endnu engcmg „ Nei . " Hvor stal der da komme nogen Enighet » fra ? Forlovelser , der sluttes i Kjodet , af Kjodets Lyst eller verdslig Beregning , dele alt Kjods Lod . Alt Kjod er Gres , og al dets Yndest som Markens Blomst . Unge <b>Piger</b> , som ingen Lyst have til det stille hnuslige Liv , som elske Balsalen hoiere end Stien og Kirken , som med Dina gaae nt » for at fee paa Hedningernes Dotre , blive aldrig gode Hustruer . Herrens Huns og hendes eget Huus bor vere din Bruds kjereste Steder . Kun den <b>Pige</b> , der kan sige til dig som Nuth : „ Hvor du gaaer hen , vil jeg gaae , og hvor du forbliver , vil jeg forblive ; dit Folk er mit Folk , og din Gud er min Gud " , bor vere din Brud ; kun med hende kan du vere <b>vis</b> paa at ville leve i Fred og Enighet » . — At imidlertid en faadcm Pagt aldrig bor stuttes uden Foreldrenes Samtykke , at disfe endog bor cidfporges for Bruden felv , og at man forst maa have sikret sig deres Velsignelse , er Noget , hvorom ingen Christen kan vere i Tvivl . Det Bifald , som efter indgacet Forlovelfe afnodes dem , er intet Bifald , den Velsignelse , som da gives , ingen Velsignelse . Du bor agte deres Ord saa hoit , at du ogsaa stille ydmyger dig under deres Nei . Og vil du <b>vise</b> dem denne 2 - Ere , saa bor du allerede adsporge dem , nåar den forste Tilboielighed fremspirer i dit Hjerte , og forinden du enduu har gjort nogetsomhelst utvetydigt Skridt henimot ) <b>Maalet</b> . — Til disse forste og vigtigste Fordringer komme

<h4>925</h4>Herren aabenbarer sin Herlighed paa dem , nåar Han gjor dem salige midt i deres Elendighed , nåar Han giver dem Hovedpryd istedetfor Aste , Gledes Olie istedetfor Sorg , Lovprisnings Kledebon istedetfor en vcmsmcegtende Acmd , nåar de staae der som Retferdighedens Terebinther , Herrens Plantning til Hans Forherligelse , nåar den Halte springer som Hjorten , og den Stummes Tunge jnbler , o , hvor prediker da ikke denne den Hellig - Aands Nnndevirkning ! o , hvor blive dn ikke Herrens Gjerninger nnbenbnrede ! De timne imidlertid ogsnn predike og vidne pnn nndre Mnnder i sine Familier . De kunne slotte Huset med siue Bouner og frembere dets Ve og Vel for Herren . Ved Hjelp af egen dyb Erfaring kunne de <b>vise</b> Sostende , Broder- og Sosterborn samt Andre Salighedsveien . De kunne forkynde sin hele Frende- og Vennekreds Guds underfulde Gjerniuger . Her i Vyen ( Leipzig ) levete for ikke lenge siden en gammel blind <b>Kone</b> , som i Vantroens Morke blev en Vederkvegelse for mange torstige Sjele . Hun kunde vidne gcmste anderledes om det Lys , som har skinnet i Morket , end de fleste Mennesker med klare Oine . — I Aaret 1739 sendte Vrodremenigheden to Misfioncerer til Vestmoen . Den Ene , Albin Theodor Feders , havde sinderet ved tre Universiteter , var en dygtig Theolog , og man satte stort Haab til ham . Den Anden , Christian Gottlieb Israels , var en halt og pukkelrygget Skrceddersvend . Den stakkels Karl havde et ynkeligt Udvortes ; men med hans Indre stod det pregtigt til . Der var han ret som en Gran , strakte sig i fast Tro og barnlig Kjerlighed opad til sin Herre og havde intet andet Onste end at tjene denne blandt de arme Negre . At han selv formanede Noget , var han langt fra at indbilte sig ; han kjendte sin egen Skrobelighed ; men Herren skulde vere sterk i ham . <b>Maalet</b> for deres Reife var Oen St . Thomas . Til St . Eustatius reiste de paa et dcmst Fartoi . Derfra vilde et engelsk Barkstib tåge dem med sig til deres Bestemmelsessted. De led imidlertid Skibbrud paa Klippeoen Skrov . Storstedelen af Mcmdstabet reddede sig i Storbanden og kom ilnnd pnn Oen , hvorimod de to Missionerer mnntte blive tilbage paa Skilet . Forent » dette gik under , havde de fimdet et midlertidigt Tilflugtssted pnn et lidet Skjer . Feders forfogte nt unne Land paa en Rekke Stene , forn ragede ov over Våndet , men gled og fandt sin Dod i Bolgerne. Den halte Israels blev tilsidst trutten ilcmd paa Oen ved Hjelp af et udkastet Toug og saaledes reddet . Da tenkte Mange : „ Den gode Arbeider ligger paa Havsens Bund , og Kroblingen lever . " Og dog blev denne Krobling med sit troende , salige Barnehjerte og sit glade Vidnesbyrd

<h4>1101</h4>velsign vort Kors for os . Du naadefulte og salige Korsdrager! Amen . Mine i Herren elskede Lesere ! Der er ingen Stand og Stilling , som Mennesket stunder saa meget efter som 3-Egtestanden og en egen Familie . En stor Deel af Ungdommens Moie og Arbeide sigler til dette <b>Maal</b> . Der er Intet , forn i hoiere Grad beskjeftiger den unge Mcmds og <b>Piges</b> <b>Sjel</b> end de Forestillinger , de gjore sig om dette nye Affnit af sit Liv . Der er neppe Nogen , som ikke efter bedste Evne har udmalt sig det med de deiligste Farver . Og dog er der iugeu Stand , som berer en dybere Sorg og Smerte i sit Skjod end nettop I mange Wgtestaber begynder Kummeren allerede kort efter Bryllupsdagen . Solen er neppe stegen op , forend der legger sig morke Skyer foran den . Gledens Taarer paa Bryllupsdagen efterfolges ofte meget fnart af Sorgens Taarer , Perlerne i Krcmdfen og paa Halfen af Perlerne paa Kinderne . Og ofte er stabet en Korsets Gang fra Alteret til Graven . Hvo vil maale al den Hjertesorg og telle alle de Taarer , forn Wgtestcmden har fremkaldt ? Lader os forst betragte det Huuskors , som Gud Herren selv paalcegger 3 - Egtefellerne ! Itte sjelden er det Tilselvet , at den Ene af dem allerede bliver fyg eller begynder at strante nogen Tid efter Brylluppet . Mange Mend og Koner har jeg hort klage og sige : „ Det var kun en kort Tid , jeg havde min <b>Kone</b> , min Mand frist ; fra det forste eller andet Aar af har min Wgtestcmd veret en Lidelsens Skole , en stadig Sygepleie . " Eller Fattigdommen slutter sig allerede tidligt til det unge Par som en undstillelig Folgesvend. Eller Herrens Hacmd hviler tungt paa Bornene . Tidligt tager Han dem bort , eller Sygdom bliver deres Lod , saa at hverken Bornene kunne glede sig rigtigt i sit unge Liv eller Foreldrene i Bornene . O , kjere Lesere ! gaaer det ogsaa En og Anden af Eder saaledes , vide ogsaa I at tale om et Morke , som nesten fra Begyndelsen af har ruget over Eders Samliv , og om andre Perler end dem . Bruden var fmyttet med paa Bryllupsdagen , fan seer vel til , nt I ikke berove Eder selv Korsets Velsignelse ! Al Trengsel er en hnnrd og suur Skal , hvori Gud har iutesluttet en kostelig og fod Frugt . Al Nevfelfe fynes , medens den er nervcerende , itte at vere til Glede , men til Vedrovelfe ; men siden giver den igjen dem , forn ved den ere ovete , Netferdigheds falige Frugt . Jesus Christus , som har gacet uuder Korset og udknnret det til siv Fane , vil under Korset drage ind til Eder . I det nye Testament feer du ret tydelig., hvorledes Trengselen maa aabne Doren for Ham .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014091526005'target=_>, 1840, Evangelisk-christelig Psalmebog : tilligemed Collecter, Epistler og Evangelier, Vor Herres Jesu Christi Lidelses-Historie, Kirke- samt andre Bønner, til Brug ved Kirke-og Huus-Andagt</a><h4>380</h4>paa dig , O ! da stal intet flrcekke mig . Hvad end mig vederfarer . Paa dig jeg har min Tillid sat ; Thi du i Sorgens skumle Nat Mig tryggelig bevarer , Mig vogter og forsvarer . 2. Her veed ) eg , Intet <b>moder</b> mig , At det jo kommer . Gud ! fra dig . Du alle Ting regjerer . Jeg da ei blot i Lykkens Skjsd , Men selv i Modgang og i Nsd , Din Styrelse vil cere . Og taalig Korset bcere . 3. Ved Korset pwver du min <b>Sjel</b> ; Det er jo da mit sande Vel , Og mig til Dyden leder . Og naar jeg Seier vundet faaer , Og hisset for din Throne staaer . Din Vusdom jeg tilbeder I alle Evigheder . 4. O ! lad mig da med Haab og Taal Bestandig see til dette <b>Maal</b> ! Det Syn mit Mod opliver . Da har jeg Trest i Modgangs Stund , Da har jeg Kraft i Dsdens Blund ; Hvo i din Haand sig giver , Han uforfærdet bliver .

<h4>2200</h4>En kjoerlig <b>Moder</b> gav du mig , En god , en trofast Fader ! Jeg kjender ikke Livets 35 ci ; Men deres Raad og <b>Lcerdom</b> ei Paa Beien mig forlader . 2. O Gud ! lad mig betcenke ret , Hvad fra mit fsrste Aandedrcet Jeg deres Omhu skylder ; Da rsrcs jeg saa inderlig , Og Kjcerlighed til dem og dig Min hele <b>Sjel</b> opfylder . 3. Mit rsrte Hjerte bryder ud I Bsn til dig , algode Gud ! Den Svage kan du styrke . Giv , jeg dem lyder redelig ! De lcere mig at kjende dig , Og trofast dig at dyrke ! 4. Lsn du dem selv med Miskundhed , Og din Velsignelse udbred Til hvad de foretage ; Og lad dem sinde Trsst hos dig , Naar Alderdommen ncermer sig Med skrsbelige Dage ! 5. Lad Dydens Gloede , Haab og Roe I deres Hjerter altid boe ! Du stedse hos dem voere ! Og , naar fra mig du kalder dem Til gode Sjeles rette Hjem , Lsn dem med evig Wre !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111903002'target=_>, 1856, Prædikener til hver Søn- og Festdag i Aaret</a><h4>876</h4>Vil man nu med Hensyn til et storre eller mindre Christenstmfund eller til dets enkelte Lemmer understge , om dets Salighedshaab er begrundct, da behovc vi altsta itte , hvor Enhver stm Bmn hm modtaget den hellige Daab , at sporge : „ er Du dobt ? " men det Sporgsmaal , vi onste beståret , er dette : „ Hm du i Forening med din Daab , Tro ? " Bekreftende vil , testovern ! aldrig Svant kunne lyde , hvor Sporgsmaalct rettes mod et hcelt ydre Kirke samfund , imod en hcel Menighed ; thi overalt er der samt Klinte blandt Hvcdcn , overalt gjelder den Sandhcd: „ Mmlge ere kalvede , men Faa ere udvalgtc . " Ja ! kunde man end sige og vidne : „ Samtlige vide . paa hvem de stulle troe , Samtlige have aflagt den gode Bekjendclst for mange Vidner , Samtlige kjende fra Barndommen af den hellige Skrift , som fan gjore viis til Salighcd ved Troen paa Jesum Christum ; " ja kunde man end vidne : „ Der fintes end itte en Eneste , som modsigcr de Troesstndhedcr , hvorpaa han engang svoer Tro og Bekjendclst , der sintes ingen eneste Troesforncgtcr og Gudsbcspottcr , Smntligc cre ustraffelige for Loveu og erbarc i Vandel , " i Sandhcd , dette er dog itte et tilfredsstillende „ Ia " pna Sporgsmaalct : „ Hm du Tro ? " Fylvestgjort er itte derved Fordringen: mig din Tro af dine Gjerninger ! " Livet , det rette Liv , kan med alt Dette vere borte , det hele Samfund kan med alt Dette vere som et dodt Lcgcmc uden <b>Aand</b> , Gjerningene kun dodeGjerningcr , itte Troens Frugtcr . Jeg kan nemlig vide , paa hvem jeg stal troe , og dog itte troe . Jeg kan nfleggc ten gode Bekjcndclse med Munden og dog itte give et levende Samtykke dertil i Hjertet . Icg kan kjende den ganste Skrift , som er nyttig til <b>Lerdom</b> , til Overbeviisning , til Rettelse og Optugtelse i Rctferdighed, og dog itte lade mig overbevist , rette og optugte af den , dog isse anvende den til min sande Nytte . Jeg kan give Troesstndheden et koldt Medhold ved isse at modsigc dcn , da Sagen kan vere mig str ligegyldig, til at jcg for dens Skyld vil forstyrres i min dorske Ro . Jeg kan leve ustraffelig for Loven og ven erbar i Vandel , isse af Wrefrygt for Gud , men nf Frygt for Straf og Ekjendstl i Vcrden , eller for at ens af Menneskene og vinde timelige Fordele . Men- hvor dette er Tilfeldet , er der Tro , sand Tro ? Hvad mangler dn ? Den cananeiste <b>Qvinde</b> , hvis Tro var stor , besvare os det . Hun var i Nod , og i si » Nod flyede hv » med Tro og Tillid he » til Jesus ned Von om Forbarmelse og Hjelp . Den Christne kan her af den hedcnste <b>Qvinde</b> lere , at han maa fole sig » sdlidc » dc , og ha » kan lerc , til hvem han kan gaae

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014110428003'target=_>Francke, August Hermann, 1879, Huus-Postil, eller Prædikener paa alle Søn- og Fest-Dage i Aaret</a><h4>520</h4>Den forste <b>Lcrrdom</b> er denne : Mange mene , at de have Christum , og have ham dog ikke . I Terttn hedder det : Barnet Jesus blev i Jerusalem , og Joseph og haus <b>Moder</b> vidste det ikke . Der mente de i Begyndelsen , de havde ham hos sig , og de havde ham dog ikke hos sig . Hvad der nu er skeet i det Udvortes , har sin Lighed med det , at Mange mene , som vi have sagt , at de have Christum , ere blevue delagtige i ham , hore ham til , kjende ham , ere i ham , da dog alt dette , naar det bliver underssgt , ikke befindes at vcrre Sandhed . Jeg har med god Betamksomhed sagt : Mange mene . Thi scmdcm det desvcrrre til med de Fleste , som kalde sig Christne . Thi eftersom de Udvortes hore til Christendom , saa forudscrtte de stedse , at de allerede have Christnm , hvori de alligevel fare saare meget vild . Thi at have Christum , er ikke saa meget , som blot udvortes at bekjende sig til den christelige Religion ; ikke heller saa meget , som at bekjende Christum med Mundeu ; men det hedder , at have Christum : at man i Gjerningen er forenet med Christo i Troens <b>Aand</b> , er delagtig i hans <b>Aand</b> , hans Liv , hans Lys og hans Kraft . Det , er bleven en Mode hos Mange , at , naar de seer En ligge van sit Iderste , saa sporge de ham , om han har Christum i Hjertet? Det Sporgsmaal er i sig selv ikke galt ; men deri fare Mange vild , at de mene , naar man har en Tanke om Christo , saa har man ham allerede i Hjertet , da dog Skriften taler meget auderledes derom , og udkrcrver , at Christus skal bo formedelst Troen i Hjertet ( Eph . 3 , 17 ) ; hvilken Christi Iboelse i Hjertet nodvendig forer med sig , at Mennesket maa have et oprigtigt Had i sit Hjerte til alt syndigt Vwsen , derimod en ret Hunger og Torst efter Retfærdighed , en fand af den Hellig-Aand virket Kjcrrligheo til Christum , og en oprigtig og " inderlig Begjcrrlighed efter at vcrre evindelig hos Christum . - Men nn findes det desvcrrre , at mangengang , hverken den , der spsrger den Anden : Har du den Herre lesum i Hjertet ? ikke heller den , der bliver adspurgt , ved eller forstaar , hvad dette Sporgsmaal har at sige . I Aab . 3 , 9. tales der om saadanne , som sige sig selv at vcrre Isder , og ere ikke , men lyve . Altsaa ere der og saadanne til , som sige sig selv at vcrre Christne , og ere ikke , men lyve . Mange troste sig i Utide og gjorde bedre , at de forst begyndte at ssge Christum , forend de mente , de alt havde ham , da de dog aldrig have fogt ham alvorlig . Christus sagde ( Joh . 9 , 41. ) til Phariscreruc : „ Vare I blinde , da havde I ikke saa stor Synd ; men nu sige I : Vi see ; derfor bliver eders Synd . " Just ligesaadcm maa man nuomstunder sige til Mange : Dersom I erkjendte , at I endnu ikke havde Christum , saa kuude I blive hjulpne ; men nu I sige : Vi have Christum, da I dog ikke have ham , bliver eders Synd . O ! at Menneskene

<h4>2520</h4>ja han havde lagt til , at det gjorde Hedningerne , de tragtede efter alle saadanne Ting ; og derpaa siger han nu : Men soger Guds Rige , og hans Retfærdighed , hvormed han vil sige saameget : Men I Me saaledes ; just deri maa I , som mine sande Disciple , vcrre forskjellige fra Hedningerne , at man sinder et ganske andet Vcrsen hos eder , nemlig , at , ligesom hines Sogen sigter til det Jordiske og Synlige , saa er eders Sogen rettet paa det Usynlige og Himmelske , og , nåar Hedningerne sporge eder : Hvorfor soge I da ikke ogsaa alle . saadanne Ting , som vi ? I da kunne svare dem : Vor himmelske Fader ved , at vi have alle disse Ting behov , han sorger for os ; men vi soge noget bedre , nemlig Guds Rige , og hans Retfærdighed . Det Eftertryk i Christi Tale maa man med al Flid lcrgge Mcrrke til . Vantroen vilde gjerne vcrre en saadau Fortoller over Christi Ord , at man vel forst og for alle Ting maa soge Guds Rige , og haus Retfærdighed ( thi det maa endog Vantroen for Skams Skyld bekjende ) ; men faa sporger den ogsaa derucrst : Hvad stulle vi crde ? Hvad skulle vi drikke ? Hvormed sknlle vi klcrde os ? Men Christus <b>moder</b> Vantroen i dens Tanker , og siger : Ikke saa , o Vantroe ! en saadan <b>Lcrrdom</b> giver jeg ikke mine Disciple , men saa siger jeg : Soger forst Guds Rige , og hans Retfærdighed , og nn siger jeg ikke : men derncrst det Timelige ; men saaledes : saa stulle og alle disse Ting tillcrgges eder , alle disse Ting , som nemlig Hedningerne soge efter , og hvad der er fornooent til Fode og Klcrder , det er , nåar I adlyde mig i det Forste , nemlig alvorligen at soge Guds Rige , saa skulle I ikke have det Andet fornoden , men det stal tillcrgges eder , eller forlenes eder som en Tilgift af Gud , paa hvilkeu I rette eders reue og enfoldige Troes Vie ; ser ikkun til , at I med eders Tro kuune fatte , ikke den menneskelige og egen Retfærdighed , men Troens Retfcrrdighed , eller den , som kommer ved Troen , og overgive Gud eders ganske Hjerte , at han maa herske deri , og regjere det ved sin <b>Aand</b> , men I tjene ham , som hans hare Born , i Fred og Glcrde .

<h4>2659</h4>som endnu staa i deres naturlige Tilstand , og ikke ere blevne delagtige i den Hcllig-Allnds Oplysning , ere ligesaa blinde og nvioende i Loven og Evangelium, som de Skriftkloge , Pharisceerne og Sadduccrerne huve vcrret til deres Tid . Der er ingen Forskjel paa eder , enten I ere Unge eller Gamle , Enfoldige eller Kloge , <b>Kvinder</b> eller Mcrnd , Lcerde eller Ulwroe . Hvo som ikke har af Hjertet omvendt sig til Gud , men fremturer i sit übodfcrrdige Boesen , han er fattig , elendig , nogen , blind og blot , han maa tykkes sig at virre san <b>vis</b> , som han allermest vil . Og de Lcerde , som endnu maugle retskaffen Omvendelse, stikte netop i det allergrueligste og farligste Bedrageri , ligesom dengang Phariscrerne vare de blindeste , fordi de havde tcenkt , og tillige sagt , de saa ( Joh . 9 , 41. ) . Ligesom dengang de Skriftkloge snakkede og disputerede meget om Loven , ja og mente , de vare meget forstandige i Materien om Christo , som Israels Folk forventede ; ligesaa er der ogsaa i vore Dage nok af Saadanne, og iscer paa Universiteterne , der , som Luther siger i sin Fortale over Epistelen til de Romere , ville va ' re kloge at domme om Troen og gode Gjerninger, og ere de storste vcrre kunde : Saaledes er der og Mange iblandt eder , der vel ikke hedde Lcrrde , men som dog , fordi de saa lcrnge have hsrt Guds Ord , tykkes , de ere meget bekvcrmme til at domme om Loven og Evangelium , der ikke vide , hvorledes ogsau de ere saa fattige , elmdige , nsgne , blinde og blottede . „ De lwre altid , og komme dog aldrig til Sandheds Erkjendelse", scmsom Paulus siger om Saadanne ( 2 Tim . 3 , 7. ) . <b>Kundskaben</b> have vel Nogle , men derfor give de just ikke den Hellig-Aands Oplysning Rnm ; thi fordi de ikke opvaagne af deres Syndesovn , og ikke lade sig opvockke af deres aandelige Dod , idet de elske Morket mere end Lyset , hvor kan der da siges , at Christus , det Livsens Lys , lyser for dem ? Saameget desto stsrre er Guds Vrede over dem , jo mere de foragte det sande Lys , og ligesom lukke deres Mne derfor . Derfor siger jeg eder , bedrager eder dog ikke saa jammerlig eders egne Sjcrle fra , og tanker ikke , I derfor ere gode Christne , fordi I fra Ungdommen af ere blevne underviste i den christelige Lårdom , men omvender eder fra Syndens til Retfærdighedens Tjeneste . I maa forst smage Lovens Bitterhed , det er , deraf erkjende eders dybe Fordcrrvelse og Elendighed i Synden , fole Guds Vrede , og vare ret bedrovede og sorge , fordi I have overtraadt Guds Lov . Naar I da i saadan sand Omvendelses Orden tage eders Tilflugt til Christum , saa skulle eders Synder for hans for eder udgydte Blods Skyld blive eder forladne , og I stnlle da ogsaa komme til at smage Lovens Liflighed , idet den Guds Kjarlighed , som har vist sig i Christo , bliver udost i eders Hjerter , men I derved opvakte og forte til den sande Kærlighed i Gud og eders Ncrste , og derefter „ vcere villige og lystige til at tjeue hver Mand , at lide Alting , Gud til Villie og Lov , der har bevist os saadau Naade " , saaledes som Luther taler om Sagen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2007072403001'target=_>Munch, P.A., 1849, Illustreret Verdenshistorie</a><h4>3792</h4>Agtelse , som un for Tiden vorder deres tildccl bos alle dannede Nationer . Dyd , hvori de athcniskc <b>Kvinder</b> paa den Tid maaske neppe knndc ovcrtrcrffes, ansaa Grcrkcrnc ei for nogen Egcnstab , der alene knndc give nogen <b>Kvindc</b> Fordring paa Agtelse og 3 Ercfrygt . Dertil fordrede Athcncrnc legemlige og aandcligc Indighcdcr, Skjonhcd , <b>Dannelse</b> , Elstvccrdighcd , <b>Aand</b> og Smag , og alt dette knnde Athenerindcrne ikke byde dem ; thi Koncrncs fra Mocndcnc afsondrede Lcvemaade og Dottrencs som Folge dcraf isolcrcde Opdragelsc knndc ikke danne lvvinder , der vare istcmd til ved sin Omgang at berede Mandcn nogen hoicre Nydelsc . Det Behov , der herved opstod hos Mccndcnc , gav paa den ene Side Oprindelscn til den unatnrlige Drift , der er be-

<h4>7996</h4>De Formaliteter , som omgive hele Romernes Retsvcesen , bidroge meget til at bersve det Liv og Udvikling ; thi enhver Formalitet drceber den belivende Nand . Saa uvigtige disse Formaliteter nu i og for sig ogsaa maatte synes , saamegen Vigtighed var dog udtrykkekg blevcn dem Magt , idet ' den ringeste Tilsidesatte ^ eaf dem var tilstrcekkelig til at gfsre Ret til Uret og Uret . til Net . Den retfcerdigste Proces kunde strande paa en Formfcjl . For at <b>vise</b> , af hvilken smaalig Art disse Formaliteter ofte vare , ville v : her anfsre nogle af dem : . „ . . , En ftastilt <b>Kone</b> maatte tilbagelevere det Nsglcknippe, ved hvis Modtagelse Huusholdningens Bestyrelse var hende overdraget . — En Son blev lssgiven fra sin Faders og en Slave fra stn Herres'Maqt, idet man drejede ham om og gav ham en svag Munddast . — Prcescription blev afbrudt ved at ssnderbryde en Gren , — Den knyttede Haand var Tegn paa en Pantscettelst eller et Depositum ; at give den hsjre Haand var et Tegn paa Tiltro . — Ved enhver Betaling maatte <b>Maal</b> og Vegt vcere tilstede som Symbol paa Rigtigheden . — En Arving , der modtog et Testamente, maatte til Tegn derpaa knepse med Fmqrene, dandse og springe . — Hsjst forunderlig var Formen for en Civilklage . Klageren maatte gribe sine Vidner i Drene og sin Modpart i Nakken og saaledes under Skrig og Klageraab kalde Borqerne tilbjcelp . Derpaa grebe de stridende Parter ligesom til Kamp hverandre i Hcenderne og traadte frem for Prcetoren , der bod dem at foredrage ham deres Trcette . Derpaa gik de igjen bort , og kom f et langsomt , afmaalt Skridt tilbage og kastede nu et Symbol Paa den omstridte Gjenstand ( f . Er . ved en Strid om en Ager en Jordklump ) ned for Prcetorens Fsdder .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008092203003'target=_>Hansen, Maurits, 1858, Mauritz Hansens Noveller og Fortællinger</a><h4>167</h4> la , her ser De nu , hvorledes De er faren , kjcrreHr . Wiger ! Det vil blive kjedeligt og stilfccrdkgt for Dem , er jeg rcrd for . <b>Mor</b> er nu gammelagtig og saa la , la ; og Thorkel tcenker nu bare paa Stald og Jord og en <b>vis</b> Tos . Jeg sidder i Vceven hele Dagen eller maa rende hojt og lavt . For en Mand af Deres <b>Dannelse</b> — ja det er ikke fordi man jo kan have <b>Dannelse</b> paa Landet ; jeg takker min salig Plejefader i hans Grav for det , han lod mig lcere og lcese . Romaner danne Hjertet , Hr. Wiger . Det tror jeg , min Plejefader erljendte , ssjsnt han aldrig sagde det , for han lcrste selv meget . Men dersom jeg kan staffe Dem en Smule Adspredelse , ere mine Boger til Tjeneste . Jeg har : Guido v. Sohnsdom , Siegsrid v. Lindenberg , de tolv sovendes Jomfruer , Fann-

<h4>172</h4>Studenten trak vel paa Skulderen og blev gansie forfcrrdet over den monstrsse <b>Dannelse</b> , og betvang sin Modbydelighed under et forlegent Smil — mener Lcrserne ? Nej , langt fra ! Jeg forsikrer dem om det modsatte . Han fandt at Gjertrud « ar en gansse fortreffelig Original ^ Stygheden og Koketteriet , Raaheden , den selvffabte <b>Dannelse</b> , hendes malpropre Dragt og dog paa en <b>Vis</b> omhyggelige Oppudsning , endelig det tillidsfulde , det hengivne i den lille HereS fortrolige Meddelelse til vor vakre , forfcrngeliae og for alt morsomt tilgjengelige Student . Det maatte jo interessere ham ! Han trykkede hendes Haand , og sagde hende snurrige Ting . Gjertrud blev gansse benauet af Velfelelse . Imidlertid glemte hun ikke , i hvilken Hensigt hun havde grebet Lejligheden ved Pandehaaret , for strar at faa en Samtale med Gjesten . Inden et Kvarteer , og lcrngere Tid turde hun ikke anvende til denne Konversation — havde hun sat ham fuldkommen ind i de herffende Forhold , naturligvis saaledes , som hun vilde have dem anstuede . Den fortryllede unge Mand , der vil egte den ssjonne Forbryderinde imod en sergende Moders og Sssters Bonner og Forestillinger . <b>Pigens</b> Koketteri , hendes Forcrldres Egennyttighed — hun glemte ikke at virkke Forhaabninger for ben lyttende Ungling om selv at vinde , idet han befordrede Gjertruds Plan til at rive Thorkel ud af den ulykkelige Forbindelse . Denne Plan

<h4>3357</h4>saa og Fader : da vil hun ogsaa efter forubgaaende , imellem os gjort Aftale have ftrget for din mere <b>Dannelse</b> , end ben , blot her kunde bekommes; enten Du nu hos den velcrrvcerdige Hr. Wiger eller ved andres Bistand i nogle Aar tilvender Dig udi adffillige Ting den Indsigt , som kan vcere Dig som Mennesie og Borger , som Kristen og Landmand nyttig og angenem ; saa tcenker jeg Dig at vende tilbage til den lidet glimrende , men derhos og lidct bekymrende Levemaade , Vcrr da din fromme Moders fromme Spn ! Glcrd hende ej blot ved et crrbart og semmeligt Liv , men ogsaa ved ssnlig Kjcrrligheds Lon ! Bild Dig ikke ind , der Du haver lcert at bogstavere , belcrgge dine Ord , kjende til Jordens Leb paa Himmelen , og Menneskenes Fcrrdsel paa Jorden , — at Du derfor maa synes ringe om din <b>Moder</b> , hvem saadant lidet angaar. Tcenk heller paa , at Alderdommens Erfaringsfrugt , samlet as et fornuftigt og retsindigt Mennesie under Livsens mangehaande Omsiiftelser, indeholder en <b>Visdom</b> , som mange Folianters tvetydige Belæring lidet naaer op imod . Og paa det jeg kan komme din Ungdom udi dette Stykke til Hjelp , saa hust paa , at din <b>Moder</b> , som maaffe vil tykkes Dig enfoldig , har havt Vid og Forstand og <b>Lærdom</b> nok , for at verre din Fader til Glcrde og Gavn , i Sorg og Savn , baade udi onde og gode Dage , endog han vel havde mer seet og lcrrt , end i din Lod falder at se og lcrre ; hvilket alt vist ikke var ham nodvendigt eller nyttigt, mens som dog giver Ret til at kalde sig en Mand udaf ' nogen 15>u-6itic>!>. Dersom Du vil se et Menneffe , hos hvem ikke er befundet Svig , dersom Du vil betragte en Sjcel , som , fraregnet sin Salighedssag, hvori den haver nok i sin Frelser , for Nesten er sig selv nok , endda den ej med boglig Klpgt er syldt , dersom Du vil fatte for Bjne et Gemyt, som heller giver end tåger , som heller betjener eller betjenes , er en Herre over sit Sind , men end Trcrl af sine Pligter , dersom Du pnsser at samle Trcekkene til et Menster for <b>Kvinden</b> , som <b>Moder</b> og <b>Husmoder</b> : saa betragt med senlig Stolthed din <b>Moder</b> ; <b>vis</b> hende din Beundring , ikke i Ord men udi Gjerning ; < rr hende i dit Hjerte ! Skulde Du , som Gud i Naade afvende , aldeles have Behov af denne faderlige Formaning, da , ak ! da vil den lidet ftugte . Men liaver Du Hjerte og Forstand

<h4>3931</h4>det ; at han derimot » blandede norsie Ord i sit Engelsk og ftrar rsbebe Bekjendtssab til det norske , kunde vor « vidende Vondefamilie ikke falde paa , men tog <b>Pige</b> for ligesaa godt Engelsk som lnou ; ja det faldt ikke engang Tone ind at undre sig over , at hendes Engelsmand forstod alt , hvad hun talte . Men nu kom Forandringen ; Dag for Dag blev hans Tungemaal mere ligt det norsie , ja inden Sneen havde astest Hoftbloden kunde Tone og Olme , der daglig omgikkes ham , kun af Udtalen af enkelte Ord mcerke at lian var Udlcrnding . Lcrseren maa ikke tro at Jakob Lommer var noget overordentligt Sprogingenium . Nej Aarsagen var ganske simpelt den , at den unge Mand var indfsdt Norsk og kun under sit flereaarige Ophold i England havde lagt Modersmaalet paa Hylden . Da hans boglige <b>Dannelse</b> egentlig var temmelig ufuldstcrndig , kom han uden at vide hvorledes , ligesom ved en umiddelbar Erkjendelse , til Kundsiab om sit Modersmaal . Med lakobS Tankegang og Selvbevidsthed stod det ikke saa vel til . Med en <b>vis</b> Angst for sig selv grundede han ofte over , hvorledes det egentlig havde fig med hans Person . Han havde ikke Mod til at sporge om , hvorfra Tone vidste hans Navn , ja han kunde ikke gjere sig fortrolig med , at han dog vel maatte vcere Matrosen Jakob Lommer , men antog oftere , at han var en ganske anden Person . Omhyggelig undgik baade Tone og ifelge hendes Instruktion ogsaa Olme at omtale hans Sklbbrud og den ynkelige Tilstand , hvori han havde befundet fig , da de gode Mennesker havde optaget ham imellem sig . Men hvad han snart tilfulde erkjendte , var den blide , behagelige Fslelse af Kjcerlighed og Taknemmelighed til den hulde Skytsengel , hvis uendelige Omhu han skyldte sit Liv og fin tilbagevendende Sundhed . SaaledeS leb AaretS sidste Traad af Tenen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014110426005'target=_>Francke, August Hermann, 1847, Huus-Postil, eller Prædikener paa alle Søn- og Festdage i Aaret</a><h4>3021</h4>Men det fornemste vi i vor Tert have at betragte hos Elisabeth er dette , at hendes ganste Hjerte var opfyldt med vor Herres Jesu Christi saliggjorende <b>Kundskab</b> ; hun vidste ved den Hellig / <b>Aand</b> , at Maria var en Herrens <b>Moder</b> , at det , at Barnet sprang i hendes Liv , betydede , at denne Herre var noervcerende, hun vidste , at Alt , hvad Maria var bleven forkyndt af Engelen , eller den ganste Prophctie om Christo stulde opfyldes . I det hun nu saaledes selv troede paa Christum , saa fandt hun i saadan Tro ogsaa for sin Deel sin fulde Salighed , og var derover fuld af den Hellig-Aands Gloede . Ligesaa er det og Hovedsagen hos enhver Troende , at hans Hjerte formedelst den Hellig-Aand er opfyldt med Jesu Christi levende <b>Kundskab</b> , og at han troer , at det ikke stal stile paa eet Stykke af alt det Gode , som er sagt om Christo i den hellige Skrift . Alt dette tilegner da Troen sig , saaledes , at enhver Troende erkjender i Troen sin fulde Salighed , og , naar saadan Erkjendelse frembryder i ham ,

<h4>4259</h4>er den eneste Grund til alt vort Haab , ja til al vor Retfærdighed, Liv , Kraft og Salighed ; og at vi stulle „ blive retfærdiggjorte uforskyldt af Guds Naade , ved den Gjenlosning , som er i Christo Jesu , hvilken Gud haver fremsat til en Naadestol formedelst Troen i hans Blod , for at <b>vise</b> sin Retfærdighed ved de forhen under Guds Langmodighed begangne Synders Forladelse ; for at <b>vise</b> sin Retfærdighed paa den ncervsrende Tid , at han maatte vcere retfcerdig, og retfcerdiggjore den , som er af Jesu Tro " ( Rom . 3 , 24 26. ) . Er det ikke en falsk Christus , naar man troer , at Christus er kun kommen til at forlade Synderne , men ikke tillige til at rense os fra al Uretfærdighed , ( 1 Joh . 1 , 9. ) , og at „ forloft os fra al Uretfærdighed , og rense sig selv ct Eiendoms- Folk , som stal vcere nidkjcert til gode Gjerninger ? " ( Tit . 2,14 . ) naar man vil bruge Christi Fortjeneste , og den Guds Naade , som os i ham tilbydes , til Meerlighed ( lud . B . 4. ) , og til Ondskabs og Hykleriets Skjul ? Er det ikke en falsk Christus , naar man ikke ligesaavel vil have Christum til sin Pr op het og Lcerer , til sin Konge og Regjerer , som til sin Vppersteprcest og Forsoning , ligesaavel have ham til Biisdom , Helliggjorelse og Gjenloselse , som til sin . Retfærdighed ? ( 1 Cor . 1 , 30. ) Er det ikke en . falsk Christus , naar man negter , forkaster og forhaaner Jesu hellige <b>Lcerdom</b> med Ord eller Gjerning ? naar man mener , at man kunde nok vcere hans Discipel , „ uden at hade Fader , <b>Moder</b> , <b>Hustru</b> , Born , Brodre , Sostrs og tilmed sit eget Liv " ( Luc . 14 , 26. ) , „ uden at forsage alt det man haver ? " ( B . 33. ) naar man tcenker , man kunde nok komme efter Christum, „ uden at forncgte sig selv , tage sit Kors op daglig , og folge ham " ( Luc . 9 , 23. ) ; man kunde nok vcere den Hoiestes Barn , „ uden at elste sine Fiender " : c . ( Matth . 6 , 44. 48. ) , man kunde nok „ tjene og behage baade Gud og Mammon , baade Gud og Verden , " fljont Christus siger , det er umuligt ( Matth . 6 , 24. fgg . ) ; man kunde nok komme til Livet og Salighed , „ uden at gaae igjennem den sncevre Port og paa den trange Vei " ( Cap . 7 , 13 , 14. ) ; man kunde nok komme ind i Himmeriges Rige , „ uden at gjore Faderens Villie , " som er i Himlene ( B . 21. fgg . ) ; man kunde nok vcere en klog Mand , og havde derfor dog ikke bygget sit Huus paa Sand , eller bedraget sig i sit Haab om Saligheden , „ endstjont man ikke gjorde efter Jesu Ord og Lårdomme" ( B . 26. fgg . ) ; man kunde nok arve Guds Rige , „omendstjont

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009091503030'target=_>Pressensé, E. de, 1866, De tre første Aarhundreders Kirkehistorie</a><h4>315</h4>lykkelige Vishnn er Gudenes Gud , „ Uuiucrscts Aarsag og Virkning , <b>Sjel</b> og Herre " ( Bhagavat-Puranas , oversat af Burnouf , 3 die Bog , 7 de Cap ) . Denne materielle Verdett har imidlertid ingen Virkelighet » . Naturen cr stjsn , vidunderlig stjsn , uten den er ikke destomindre den evige Maia , den skuffende Lsgu , den trolsse Bolerstc , hvis Omnrmelsc mnn man flye . „ Som dett stjsnuestc <b>Qvinde</b> fremtraadtc hun ien Skov , bvis Treer vare bedekkede med mangeslags Blomster og guulagtige Knopper . Det syntes , sont hendes stante Legeme maatte gaue midtouer , nåar hun flyttede fite lette Fsdder , der lignede sine Knopper . Hendes store Vitte saae nd som beuegeligc Stjerner ; hendes trollede Haur forstjsntede hetttes Ausigt . Med sin fine Harmt » luftede hun fin Kledning fru Jorden . Den fortryllende Henrykkelse , huormed hun uur omgivet , bragte Forvirring i Verden " ( Bhag . Pnr . Bdc Bog , 12 te Cap ) . Denne fortryllende Main er det materielle Liv , Stnbningernes Elendighet» og Treldom . Fullkommenheten bestod deri , at nem unddrnger fig hende ved Bod , og Boden maa drites snauidt , forn det stete i Buddhismen. „ Virtfomhedeu gjsr , nt Menuestct lener , Uvirksomheten sitlrer det Udsdelighed . Eneboeretl , som seer Sandheden , stal opoffre Skuffelsctl i den Gcvidsthcd , at han har Aanden ; nvirksom stal hun hvile i slantens Skjsd ; enhver af de Grundbcstnnddele , hvoraf huus Lcgcutc bcstnner , stal hnlt igjeu lade vende tilbage til dens Plads ; hun stal lade Jorden vende tilbage til Vundet , Våndet til Lyset , Lyset til Vinden , Vinden tit Mtheren , Mheren til Personlighedens hsiere Grunduescu, Pcrsoulighcdeu til Forstaudcu , Forstanden til Naturen , og Naturen til den übevegelige Auud ; uaar han snnletes er rykket nd nf Toleden, gnner han ud sot » Ilden , uaar den ikke lengere fater Nering . Vifhnu er iucarneret for ut fsrc Meunesteuc til dette uåndelige Liv . Hnns Incarnationcr huve mnngfoldiggjort sig i det Uendelige , fordi det phyfistc Liv ingen Virkelighet » hnr og Individualiteten er uden Verd . Hun vifcr sig snart i et Menneskes stussende Skikkelse , snurt forn en Fisl , et Vildsviin eller en Lsue , snart som en Dverg eller en Helt " ( Bhag . Pnn , 7 te Bog , 12 te , 14 de og 15 de Cup ) . „ Det hsieste Vesen cmtagcr eller forlader hsie eller lavere Legemer , hvis Attribnter ere Hjertet , Sandsene og Elementerne ; det gjsr han ved sin > gen Magt og vedbliver at uere forstjellig fra disse Legemer . " Isute Incarnationcr ligner han en Skuespiller , der huert Vedlik ifsrer fig en ny Forklcrdtting. <b>Formaalet</b> for hans Incarnationcr er klart angiver pnu fslgende Sted : „ Det ustubte Vesen forlader det Legeme , hun har betjent sig uf , for at befrie Jorden fru den Byrde , der trykker den , ligesom mun druger en Torn til at trekke en anden nd med " ( Bhag . Pur . , Iste Bog , 15 de Cap ) . Tornen er det materielle Liv , hvilket Vishnn tun tilsyneladende

<h4>581</h4>kun ikke en reen <b>Sjel</b> . Ikke fornsiet med at have Elskere af samme Stand som hun selv , ssger Patricierinden dem i Folkets Berme , mellem Slaverne og Gladiatorerne . Undertiden saae man endog <b>Qvinder</b> kjempe paa Arenaen . let Maleri as forferdende Skjsnhed fremstiller luuenal ( 6 te Satire ) os med eet Trek Frekheden hos sin Tidsalders <b>Qvinde</b> , idet han lader hende gaae smilende forbi Kydsthedens Alter . Clemens fra Alexandria skildrer os vel den hedenske <b>Qvinde</b> med en kydstere Pensel ; men den Forestilling , han giver os om hende i sin Pedagog , svarer fuldkomment t ' l luuenals sjette Satire . Kostbart kledt , oversst med vellugtende Vande , sterkt sminket nsies hun ikke med de ussmmelige Malerier , der opfylde hendes Bolig ; hun lader dem endogsaa afbilde paa sit Fodtsi . Hun lever midt i stjendselsfuld Overdaadighed , snart beskjeftiget med forfengelig og fordervelig Snak , snart aabnende sit Bre for gamle Koblerskers Raad , snart omgivet af Gjsglere og sjeldne Fugle ; til andre Tider lader hun sig bere ud i Byen ien Berestol , begiver sig til de offentlige Bade eller til Udsalgsstederne, hvor Lediggjengerne strsmme sammen . Natten tilbringer hun med Fester , hvor man lader enhver Tilbageholdenhed fare og endog gaaer saa vidt , at man beruser sig . Man kunde kalde hende en Perfonification af LEgtestabsbrud . Denne smukt udpyntede <b>Qvinde</b> , „ligesom Venus bunden til Lasten med gyldne Lenker " , skjuler ogsaa med sit glimrende Idre den skjendigste Fordervelse , i Lighed „ med de tilsyneladende pregtige egyptiske Templer , som i deres inderste Helligdom skjule en heslig Guddom " ( Clem . Alex . Pedag . , 11 , 2 , 33 ; 111 , 2 , 4 ) . Med Hensyn til Mendenes Seder vil det neppe vere muligt at give endog kun en Forestilling derom . Unaturlige Laster , det greske Hedenstabs ulegelige Saar , havde uden Skranke og uden <b>Maal</b> nduiklet sig i Rom . Alle Classer af Samfundet vare smittede deraf . Da Vellysten altid fsrer Grusomheden med sig , see vi Foreningen mellem Udsvevelser og Mord , vi have paauiist i Naturreligionen , trede frem efter en storartet Maalestok . Tacitus forteller ( Ann . XIII , 44 ) , at en Romer paa hans Tid endte en Nat , der var tilbragt i Orgier , med at myrde den Skjsge , som havde spillet Hovedrollen ved Udsvevelserne . Denne Blanding af Lyst og Blod er eiendommelig for hele Keisertiden . Derfra hidrsrer den Indest , hvori Legene paa Circus staae ; Skjsgerne boe tet ved Arenaen , hvis Sand drikker Gladiatorernes i Strsmme udgydte Blod . Man nsies ikke lengere med Slaver ; man lader ogsaa Soldater, ja Centurioner kjempe sammen . Om Morgenen begiver man sig til Circus ; man bliver der om Middagen , i Mellemtiden mellem Kamplegene ; om Eftermiddagen give Naumachier eller Ssstag paa et hurtigt tilveiebragt Hav Anledning til Hundreders Dsd . Den stsrste

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013121306010'target=_>Draper, John William, 1875, europæiske Aandsudviklings Historie</a><h4>521</h4>Da der var Tale om Vedaismen , berørte jeg den Maade , hvorpaa dens ophøiede Forestillinger efterhaaden bleve fortrængte af et lavere Trins Forestillinger , der traadte i Forgrunden ; og det vil ogsaa her være formaalstjenligt paa lignende <b>Vis</b> at tale om den tilsvarende Forvanskning af Buddhismen . Dens praktiske Virkning yttrede sig i Indførelsen af et storartet Munkesystem , der frembyder mange Lighedspunkter med det senere Europas . Da dens <b>Formaal</b> ganske var af personlig Art og blot gik ud paa Opnaaelsen af indjviduel Lyksalighed , var det ikke muligt Andet , end at den maatte afføde den mest yderliggaaende Egennytte . Den indprentede Enhver , at han maatte sikre sit eget Velfærd uden Hensyn til alle Andres . Hvad bekymrede han sig om Forældre , <b>Hustru</b> , Børn , Venner og Fædreland, nåar han blot naaede Nirvana .

<h4>1427</h4>Af stor Betydning for det romerske Suprematis successive Udvikling var den Strid , som Eutykes , Arkimandrit ved et Munkekloster i Konstantinopel , vakte . Han havde gjort sig bemærket som en af Førerne for de Opstande , der forefaldt under Nestorius , og havde ligeledes spillet en fremragende Rolle under senere Uroligheder . Eutykes blev anklaget af en i Konstantinopel afholdt Synode for at nægte Kristi dobbelte Natur og for at paastaa , at der ogsaa maatte gives to Sonner , hvis der gaves to Naturer . Han overbevistes om at være skyldig og sattes i Kirkens Band . Dette var dog kun den foregivne Grund til hans Domfældelse , den sande Grund stod i Forbindelse med en Hofintrigue . Den første Eunuk , der var hans Gudson , havde dengang et dobbelt <b>Formaal</b> for øje : han vilde nemlig ophoje Eutykes til Biskop af Konstantinopel og gennem Eudoxia , Kejserens <b>Hustru</b> , tilintetgjøre den Indflydelse , Pulkeria , Kejserens Søster , var i Besiddelse af . Eutykes henvendte sig efter sin Domfældelse til Kejseren , der efter Eunukens Tilskyndelse sammenkaldte et Koncilium i Efesos . Dette var den saakaldte » Røversynode » . Dette udtalte sig til Fordel for Eutykesses Orthodoxi og forord nede hans Genindsættelse , medens det afsatte hans Anklager Eusebius og Biskoppen af Konstantinopel , Flavianus , der havde været hans Medbejler og paa hin Synode hans Dommer . En Opstand udbrod , under hvilken Biskoppen af Konstantinopel blev myrdet af Biskoppen af Alexandria og en <b>vis</b> Barsumas , der under Raabet : » . Dræb ham ! dræb ham ! « angrebe ham med Næverne . De italienske Legater undkom med Nød og næppe . Det heldige Ddfald af disse Bevægelser skyldtes væsenlig Dioskuros , Biskop af Alexandria , der paa denne Maade fik sine Medbejlere iAntiokia og Konstantinopel styrtede . Et kejserligt Edikt gav Konciliets Beslutning Lovskraft . Men her traadte Biskoppen af Rom imellem , idet han vægrede sig ved at anerkende det Skete . Han saa gjærne , at Alexandria og Konstantinopel laa i en stadig Kamp med hinanden , men ønskéde ikke , at noget af dem skulde vinde Overhaanden . Derpaa afbrød Dioskuros Forbindelsen med ham . Rom og Alexandria befandt sig nu i Strid . Da døde Kejseren paa et heldigt Tidspunkt . Hans Søster , den orthodoxe Pulkeria , Leos Veninde , ægtede Marcian og gjorde ham til Kejser . Et Koncilium sammenkaldtes i Khalkedon . Leo ønskede , at det skulde holdes i Italien , hvor Ingen havde kunnet gjore ham Rangen stridig . Saaledes som Sagen nu stod , slog han ind paa den gamle

<h4>1777</h4>Da Konstantin den Store og hans Efterfølgere under Paavirkning af Gejstligheden havde erklæret sig for Fjender af den verdslige Videnskab , maatte de Gesjtlige paatage sig den Forpligtelse at sorge saavel for Folkets fysiske som for dets religieuse Tilstand . Det stemmede ikke overens med Forholdene i Almindelighed , at Læger, der i filosofisk Henseende heldede til det hedenske Parti , kunde taales længer . Under deres <b>Uddannelse</b> i Asklepierne meddeltes der dem Ideer , som stode i Modsigelse med de nye Begivenheder . Konstantin undertrykte derfor ved et Dekret hine Anstalter , men der blev truffen omfattende Forholdsregler til at lade dem erstatte af andre , der svarede mere til Kristendommens <b>Aand</b> . I Hovedstæderne grundedes der Hospitaler og velgj ørende Stiftelser , der bleve rigt doterede med Penge og Jord . Kejserinde-Moderen Helena deltog personlig i slige humane Foretagender , og hendes Exempel fulgtes af mange højtstaaende <b>Damer</b> . <b>Kvindens</b> Hjærte , der af Naturen er aabent for de Sørgende og Forladte , giver snart sine Følelser et virksomt Udtryk , nåar det helliges af en blid Kristentro . I denne berettigede Retning kunde Kristendommen udfolde sin exempelløse Velvillie og Velgjørenhed . Institutioner af den foran berørte Beskaffenhed indførtes efter en storartet Maalestok ; nogle af dem optoge Hittebørn , andre forældreløse Børn og atter andre Fattige . Vi have i det Foregaaende omtalt Parabolanerne eller Sygevogterne og vist , hvorledes de bortlededes fra deres oprindelige Bestemmelse .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015030308004'target=_>Evers, Ernst, 1879, Rogate</a><h4>1557</h4>Manden ! og da Kristian saa og mcerkede det , da stod det fast i hans Hjerte : den Plads skulde hun ogsaa beholde ! og han var nu den rette Mand til at give det Guds Ord Gyldighed : Manden er <b>Kvindens</b> Hoved ? " Gjennem Tugten i det Hus , hvori hun havde fundet Optagelse , gjennem Barnets Undervisning og , den übevidste Indflydelse , som dets Bsn og <b>Loerdom</b> svede paa hende , varder gjort en gavnlig Begyndelse , da hendes egen Mand med saft Haand greb Tsilerne . Underststtet as sin Persons Mandskraft udsvede hans faste , alvorlige Boesen , hans flittige , strcebsomme Vandel en saa stor Indflydelse , at hun flet ikke faldt paa at scette sig op imod ham og henfalde til de gamle Daarstaber . Hun forstod og mcerkede for hvert Skridt , at hendes Mands Aine saa paa hende ; han var i Tugthuset vcennet til streng Orden og Renlighed saa undgik nu Intet i hans eget Hus ham , naar Noget manglede . Da greb han endog selv engang imellem Kost og Bsrste og saa derunder med et saa besynderligt, ncesten medlidende Blik paa sin <b>Kone</b> , at hun blev ganske rsd lige til Panden . Men det Bedste var , at han om Aftenen , naar Alt var bleven ganske stille , tog hende i sine Arme og gjorde Forbon for <b>Kone</b> og Barn og for sig selv , at den gode <b>Aand</b> maatte lede dem sammen paa den jevne Vei .

<h4>3060</h4>Brsdet , ligger der , og hun maa se , hvorledes baad . Forretningen og Husvcesenet gaar tilgrunde . " „ Det mener jeg visselig ogsaa , kjcere <b>Kone</b> , " svarede han , „ at det er en saare vanskelig Ting , og jeg vild , egentlig ogsaa kun <b>vise</b> dig , at jeg ikke af min eger Kraft dengang kunde boere Korset med Frimodighed men at det ene og alene var den trofaste GM Naade , som for den Syges Skyld gav mig detrsstige Mod ; thi ellers havde hun jo fslt fim Smerter dobbelt . Du har jo ogsaa ret havt dir cerlige Byrde at bcere med mig , og mangengang , naar jeg tcenker paa denne Tid , maa jeg undr « mig over , at vi overhovedet kunde holde ud uden a < forsage . Hvor lidet var jeg dog frist i de fsrste syd Aar af vort Wgtestab , og selv de Dage , vi kaldte Sundhedens Dage , vilde nok de Fleste regne til Sygedage. Ja , ja , den kjcere Gud har ogsaa heri bevisi os sin underbare Kraft i rigeligt <b>Maal</b> . "

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016063008035'target=_>Stowe, Harriet Beecher, 1853, Onkel Toms Hytte, eller Negerlivet i de Amerikanske Slavestater</a><h4>4444</h4>Sandt nok , der er en religiøs Fortrøstning endog for den morkeste Time . Mulat-Konen var et Medlem af den Methodististe Kirke , og omendfijont meget lidet oplyst , havde hun en oprigtig from <b>Aand</b> . Emmeline var bleven opdragen med mere Omhu , — var bleven oplært i at lcese og sirive , var flittigen bleven underviist i Vibelen af en trofast og from <b>Huusmoder</b> ; og dog — kunde det ikke friste selv den fasteste Christens Tro at sinde sig selv tilsyneladende forladt af Gud og voldgivet den übarmhjertigste Grusomhed? Hvor meget mere maa det da ikke ryste Troen hos Christi egne arme Smaae , der ere svage i <b>Kundskab</b> og unge af Aar !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011062923007'target=_>Pontoppidan, Erik, 1880, Menoza, en asiatisk Prins som drog Verden omkring og søgte Kristne, især i Indien, Spanien, Italien, Frankrig, England, Holland, Tydskland og Danmark, men fandt lidet af det, han søgte</a><h4>931</h4>og den Herres Tjeneste , man stod i , ja . fra al Leilighed at gjore Andre til Gode , eller at hjelpes af dem i Nodsfald ? Fader lochum svarede : Erindrer Eder , at jeg tilforn sagde , min Levemaade blev begyndt , da jeg endnu kjendte lidet til Gud , og altsaa ikke uden nogen Vildfarelse ftaa min Side , stjsndt jeg er <b>vis</b> paa , Guds <b>Visdom</b> har seet til for mig usle Daare , og styret mig endogsaa ved en selv udvalgt Afvei og Omvei til sig selv , som er Vcien , Livet og Sandheden. Herncrst vider ogsaa , at jeg var den Gang 53 Aar gammel og havde gjort Gavn efter min Stand fra Ungdommen as . Min <b>Hustru</b> og mine Born vare dsde . Jeg var kjed af mig selv , og af al Verden . Jeg aabnede mit Anslag for min Skriftefader , en Barfod-Mnnk i ncrste Kloster , og da han ikke alene samtykkede , men endog roste det samme , saa var jeg snart fcrrdig . Endnu er jeg ikke heller ganske vrkeslss. Her ser I min Haandten , paa hvilkcn jeg daglig spinder lidct til Gavn for mine Brodre .

<h4>3195</h4>stere , og Andre fandt paa noget Andet , som skulde betegne stor Ydmyghed og <b>vis</b> Udgang as Babel . Med disse arme Menneskcr maatte han have Medlidenhed ; thi det er venteligt at de Fleste have havt et godt <b>Maal</b> for Die , nemlig deres Sjcrls Frelse , endskjsndt de , af Mangel paa Indsigt, lode sig overile til at bruge saa daarlige Midler . Hvad Klcrdedragten angaar . da maa jeg tilstaa , at København er et anstsdeligt Sted for den , der kan stode sig over Sligt ; thi der vil den Ringeste , om han har Evne ikke vige den Fornemste i Jeg mindes , at da jeg fsrst kom til Kjsbenhavn , syntes mig , jeg aldrig havde seet saa mange Standspersoner samlede paa et Sted ; thi som saadanne ansaa jeg ofte Hacmdvcrrksfolk , iscrr dem af Kvindekjsnnet , og hvem stnldc mene andet , end at et <b>Fruentimmer</b> , forn kjorer til Kirke , vel udstafferet og bchocngt med LEdelstenc , var af adelig Stand , ikke at de kunde vwre en sict og ret Person , ofte en Kjobmands eller en anden Borgers Tjener , som bar guldbroderct Vest og Damask til sine Klcrders Underfor . Om denne Materie talte jeg engang med Hr. Schr . ~ som ogsaa snskede , at dette var anderledes , men sagde derhos, det var vanskeligt at stoppe alle Ndflugtcr for Sindcts Forfcrngelighcd , hvilken ikke faa snart hindrcs paa en Side , forend den bryder ud paa en anden , ligesom Bylderne paa et Legeme , fuldt af Usundhed ; saa at den eneste grundige Kur maatte ske indvortcs i ethvert Mennesics Sind og Hjertelag, efterdi alle udvortcs Ting dog ikke have anden Moralitet, end den Hjertes Hensigt giver dem , og at overalt , den ene Daarlighed er saa god som den anden , hvad Samvittighed angaar , endskMdt i Hcnsecnde til Omkostnmg og det borgerlige Vcrsens Velstand , en Mode kan verre mere landsfordcervelig eud den anden , og derfor : en pollttsk ,

<h4>3418</h4>Over den underlige Tummel i Verdens Niger , over Kirkens Uroligheder og Anfcrgtelser , snart af et , snart af det andet Slags Fiender , fcrldes samme Dom , nemlig , at Enden og det yderste <b>Maal</b> blive snskelige , efterat alle Tings stridige Dele noksom have svcrrmet , gjcrret og gjennemtrcrngt hverandre . Imidlertid bor Enhver af os passe paa sin Post , og gjsre sin Pligt ester allerbedste Indsigt og Evne , men derhos tro , at hvad der siden <b>moder</b> ham,-er godt , og det bedste der kunde komme , om ikke i sig selo strax paa Stand og Sted , saa dog visselig i sin Samling og Hensigt til Hovedsagen , hvilken enwu skjules , men stal aabenbares i sin Tid . Hvo som indser denne Sandhed grundig og tilgavns , samt vcrnner sit Sind til Troens Vvelse og alle Tings Overladelse i den <b>vise</b> , mcrgtige og kjcerlige Guds Haand , hans Sjcrlero , Fred og Tillid tabes Me ved udvortes Omstiftelscr , den hcrnger ikke as Hcrndelser , og tumler sig ikke tilligcmed de mange Tilfalde , som alle skabtc Ting idelig erc undcrkastede . Hans Olie svsmmer oven paa alle Vande , og ingen af Guds Forsyns underlige Omveie er ham en Afvei .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008091503002'target=_>James, G.P.R., 1843, Morley Ernstein</a><h4>134</h4>vel vogte sig for at misforstaae Udtrykket her . Ikke en eneste syndig Tanke havde sundet Plads i hans Hjerte Han var altfor ung og hans Hjerte var altfor ufordervet , til at han deri skulde bcvaret nogen Tanke , som ikke den skjonne <b>Pige</b> , han havde seet , gjerne maatte vide , og , naar hun var kommen til <b>Kundskab</b> derom , ikke skulde give sit Bifald , om man endog forudsattc , at hun besad alle de Fortrin, hendes Ansigt lovede , saa uskyldigt og stjsnt og helligt det end var . Ikke dcstomindrc ville vi vedblive vor Paastand om , at det Forgjengeliges <b>Aand</b> var seirende — den urolige, virksomme , heftige <b>Aand</b> , den der leder Loven til at styrte efter sit Bytte ; den <b>Aand</b> , der bringer Krigeren paa Valpladscn — denne Verdens <b>Aand</b> , det forgjengelige Haabs , Lidcnskabernes og Sindslidclsernes <b>Aand</b> , — den <b>Aand</b> , der i alle sine forskjellige Skikkelser i Leiren som i Staden , ved Hoffer og i Cabinetter vinder baadc hoi verdslig Bcrommelse og et vidt bckjendt Navns blendende Glorie ; men denne <b>Aand</b> er ikke andet end en dyrisk Kraft — Stovets og Forgjengelighedens <b>Aand</b> .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111103033'target=_>Wallin, Johan Olof, 1854, Prædikener over de aarlige Søn- og Helligdages Evangelier</a><h4>1555</h4>Elskede ! ikke Jorden , men Himmelen indeslutter dem ; forstaacr dct himmelske Sprog , som taler til Eders Hjerter fra en Ven eller LEgtefcelle , en Broder eller Soster , en Fader eller <b>Moder</b> i de Levendes Land : I have da vel nu Bedrøvelse , men jeg vil see Eder igjen og Eders Hjerte skal glcrdes , og Ingen tåger Eders Glcrde fra Eder . Og Du , Lazarus ! som i Nod og Elendighed forsmcegter ved den übarmhjertige Fraadsers Dor ! hav Taalmodighed , lad din Sjcel forccdlcs af Ulykken , ligcsom Guldet luttres i Ilden , og tcenk siden , ( hvad Du kun da kan tcenke ) , at Gud haver udvalgt den » , son , ere fattige i denne verden , til at vorde rige i T ^ roen og Arvinger til Aiget , O ! vi Alle , som her , fulgte af Ekrobcligheder , saarede af Lidelsen , ncdsynkcnde under Byrdcrne , nogle Oieblikke sukke i en henfaldende Hytte , vi ere altid frimodige , om vi end bsr at indgaae i Guds Rige med mange Trengsler , thi vi vide , at dersom vor Hyttes jordiske Huus nedbrydes , saa have vi en Bygning af Gud , et Huus , som ikke er gjort med Hcender , evigt i Himlene . Gnd raader baade for det Liv , som nu er , og det tilkommende og Gudsfrygt haver Forjettelse for begge . Vor Bane er uendelig ; lader ikke vore Fodder snuble eller trccttes ved disse forste Skridt ; lad ikke vor Tro forstyrres , men heller befestes af den begyndcnde Vandrings Gjenvordigheder ; lader os ikke bedomme , hvad vi endnu ikke kunne overscc , men rolige forvente , hvad den hoieste Viisdom og Kjcerlighed har opbevaret for rene Hjerter . Lader os vende Oiet med Trost fra det Sorgclige , Dunkle , Uforklarlige , som her saa ofte omgiver og ncdslaaer os : det er ikke alene en Overgang , det har en Indstydelse vaa lysere Udviklingcr , dct hensigtcr til hoiere <b>Formaal</b> , det staaer i Sammenhang med evige Skjebner . Saaledcs lerte han , som bragte Liv og Uforkrcrnkelighed for Lyset ved Evangelium , han , som ikke alene under hele sin jordiske Vandring kun virkede for Himmelens <b>Formaal</b> og i Doden befalede sin <b>Aand</b> i Faderens Hender , men ogsaa opstod af Graven , for at lceggc for Dagen , hvorledes den samme Almagt , som har frembragt Menneskelivet , ogsaa kan opretholde og fornye det ; og Enhver , som har Sands for Menneskenaturens Ophoielse , det Godes uforgjengelige Verd og den evige Orrens Nodvendighcd , omfatter med Hjertets fulde Overtydning denne hoie , gl ^ derige , Alt forklarende Sandhed .

<h4>1942</h4>end striver mod Christcndommens baadc <b>Aand</b> og Forskrift , saa skulde ct betragtende Blik paa Menneskene omkring os og en anstille ! Sammenligning med deres Liv om Sondagcn og om Hverdagene , paa en sorgelig Maade bevise , hvor mange Christne der ogsaa findes af dette Slags ; < og vi kunne vel med Grund sporge om , hvilken Nytte de have af deres Kirkegang ? — En rctsindig Gudsdyrker vced vel , at der ikke gives nogen hoiere Gjenstand for hans 2 ( srefrygt og Kjccrlighed end det hsieste , milveste Vcesen , og at en Dag , som udelukkende blev bestemt til den Eviges Dyrkelse , bor , endog uven alle andre Hensyn , holdes hellig for Herren ; men han veev ogsaa , at Pligterne mod Gud staac i et saa umivdelbart Sammenhang med Livets ovrige Pligtcr , at lagttagelsen af disse Sivste vel maa ansees for at udgjore den silkrcste Prove paa Gudelighedcn , famt at en anden Gudstjeneste , uven Indfiyvelse paa Menneskets moralske Udvikling , er baade urigtig og unyttig . Med denne Overbeviisning kommer han i Guds Huus og forstaacr da , ogsaa der , at prsve Alt , og beholde det Gode . Og godt er Alt , som ovlofter hans Sind over Jorden , Alt hvad der bibringer ham klarere Begreber om Menneskets Vcerd og <b>Formaal</b> , Alt hvad der forer ham til en mere levende <b>Bevidsthed</b> om Guds Bsrns Herligheds Frihed , Alt hvad der giver ham et rencre Sind , en mildere Folclsc , ct standhaftigcre Mod , en cedlcre Kraft , en urokkeligere Fred at vcere i Besiddelse af og forvare under Fristelserne , Farerne , Byrderne , Bcsva'rlighcderne og Verdens onde , urolige Vccsen ; ja ! Alt hvad der , eftcr den <b>Kundskab</b> han har om sig og sin Stilling , er for ham den ommeste Advarsel, stcrrkeste Opmuntring , den herligste Husvalclse , hvad der bcvst behoves og Narest opfatt ^ s . Dette modtager han af hvad han horer , betragter og erfarer ; dette nedlccgger han , trofast og glad , i sit ihukommende Hjertes Forraad , og har det siden stedse ved Haanden under sine ovrige Dage . Derfor seer han omkring sig med rolige og milde Blikke paa Verden og Livet ; derfor er han taalmodig og fredsommelig , tryg og fornoiet , trofast og redelig , huld og samvittighedsfuld , barmhjertig og rctfcerdig . Han vandrer under Guds Sine , og <b>moder</b> intet af sine bctrcengte Medmennesker , uden at mindes sin Frelsers Ord : Saameget , som I have gjort for een af disse inine mindste Brsdre , have I gjott for mig . Gjennem sin rene Kjcrrlighcd overbeviser han sig selv om sin sandb Tro , thi hvo , so , n ikke elsker sin Broder , som han seer , hvorledes kan han elske Gud , som han ikke seer ? Han vccd , at der for den Hellige ogsaa er en reen , uundgaaclig , übesmittet Gudsdyrkelse , at bessge Faderlose og < knker i deres T ^ rcengsel , at bevare sig selv übesmittet af ' verden . Med dette Indtrpk forlader

<h4>2234</h4>Nost , som vcckkcr eller advarer , opmuntrer eller forferder , opfordrer eller holder tilbage . Uvbreder Freden sine Velsignelser over Lanv og Nige , saa hores Herrens Nosi :  Foler Eders Lykke og misbrugcr den ikke . Lader ikke Blodagligheden dysse Eders Sandser isovn , Ippigheden fortcere Eders Kraft og Eders Goder ; men vcercr aandelige , arbeidsomme, eendrcegtige og forbereder Eder , medens det endnu er Tid , mod Omverlingernc . " Ncrrmer Krigen sig med sine Ncedsler , saa taler Rosten :  Vccrer ikke forsagte , men trofaste . Undertrykke ! sens Lcenker smcdcs kun for Feigheden og Selvforagten . Tcenker paa Eders Fcedre og paa det Land , de har cftcrladt Eder , ikke alene at hoste , men ogsaa at forsvare . Tcenker paa Eders Fcedres Gud , hos hvem er ingen Forhindring at frelse ved iNange eller ved Faa . " Komme Landeplager, Farsoter , haarde Tider , saa raaber denne Nost :  See , Guds Vredes Niis er udstrakt ; betcenker , bckjender Eders Misgjerninger ; omvender Eder til Herren , I Frafaldne ! og , hvad han saa siden gjor , vccrer taalmodige , varer fromme , oploftcr Eders Hjerter over Jordens baade Fordele og Tab , til det Gode , som varer cvigt . " <b>Vise</b> ' sig paa Afstand eller i Nccrheden Vantroens og Letsindighcdens ulykkelige Folgcr; oploses eller svekkes baade de moralske og borgerlige Baand , som , alene under en levende Folelse af det Hellige , sammenholde baade smaa og store Samfund , saa byder denne Nost :  Samler Eder i Andagt om de forglemte Altare ; vender tilbage til det Alvor , uden hvilket Vivet hverken har Vccrd eller Forhaabning ; fornyer og forener Eders Hjerter i den Tro , som folger af Helliggjorelse og No ; thi det er alene Religionen , som giver Scederne Helligajorclsc , Lovene Kraft og Samfundsordcnen Bestaaen . " I Alt forkynder sig den veldige Ttcmme ; tegnet i Tvorene af alle Verdens Omveltninger , staaer den store GudS Tanke : Aotfcerdighed oph ^ ier et Folk ; men Synd er en gruelig Ting idlttndt Folk . Vel dem , som opfatter den , baade af Tidernes Historie og af Tidens Begivenheder ! SaliZt er det Folk , hvis Gud Herren er ! De stulle hvile som trygge Born ved den nurgtige Aands Arm , hvor han gaacr frem uden Hindringer til sine hoie <b>Maal</b> — ofte gaacr frem saaledes , at Jorden ba ' ver og Nigcrnc sammenstyrtc . Men , ogsaa paa din egen Bane , Dodelige ! vaagcr det samme alvise Oie , omsvccver Tig samme Guddomsmagt ; og Alt , hvad der <b>moder</b> Dig , er et Vink fra den , som ikke lader Noget skce uden Hcnsigt ; Alt , hvad Du fornemmer , er en uotrykt Tauke af ham , som besluttede og ordnede Alt for dit Vel , men som overlod til Dig selv at folge eller ilke folge den givnc Anviisning . Betcenk det , Menneske ! og fol , i hvilken Grad din Tkj.rbne er lagt i dine egne Hcendcr . Gud har

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081903010'target=_>Borring, L. S., 1859, Fransk-dansk Haand-Ordbog</a><h4>124</h4>v. « . scette en Sag i Gang ; forberede Udfoirlsen af en Plan ; bruges tun ( Nan . ) vcenne en Hest til at gaae lige frem . B ' - , begive sig paa Vei ( ver » eller clv este 6 e ) ; stride frem , ncrrme sig <b>Maalet</b> . neter , v. « . kjobe ; - cze . k czn . , kjobe Noget af En ; » en FroB , en 66--«nil, kjobe i store , i smaa Partier ; - au eumptant , kjobe for rede Penge ; kere-6it, o . te , me , k . paa Kredit , for at betale til en <b>vis</b> Termin ; - puur Bon eomnte , k , for egen Regning ; - var cuinin ! BBic > n , k . for Andre ; - ener , k . dyrt ; - i » , dun inln-ene , k . til godt Kjob ; - » , vil prix , k , til Epotpriis ; - au < ! ciul > le , betale det Dobbelte ; - nu vniclB cle I ' c » - , opveie med Guld ; - cio Bnn Ban ^ , au prix cio « on « nn ^ , betale med sit Blod ; il I ' a dien ucncte , han har erholdt det med Brsvcrrlighed ; - 6 eB danB , lose Wgteskabsbevilling. ? n . - euBe , / . Kjober ; Hun » kjonsformen bruges ofte spottende om en <b>Kone</b> , der vil kjobe Alt , hvad hun faaer til . 6 , / , . fuldendt ; fuldkommen ; une deaute - cc , en fuldendt Ckjonhed ; un luv - , en Crkenar .

<h4>1693</h4>« ck . som , ligesom ; saasom , for Giempel ; ncesten , saa godt som ; hvorledes ; hvormeget ; saameget som . i Ggenskab af ; con / , da , fordi , efterdi ; > cela , nogenlnnde ; il ost - oela , saaledes er han , det er hans Karakteer ; - Bi , ret som om ; - huoi , hvorledes ( / i . u ) ; on le tiouvll , - mort , man fandt ham ncesten dod ; il ost - lo reBBoi-t 6 e « ette entrepl ! Be , han er paa en <b>vis</b> Maade Hovedhjulet i dette Foretagende ; - il reluBe 6 e pn ^ or , ete . , da ( efterdi ) han negter at ville betale , saa 0. s . v. Lommeinova.iBuu , / . Erindringsfest af en Helgen ; kirkelig Mindefest for Afdode . ( ^ oininemorlvtil , ve , a . gjenkaldende en Erindring . ( ? ommLinnrlltion , / . kirkelig Ihukom » melse af en merkelig Begivenhet » ; I : » , - 6 eBinoi ' tB , Ihukommelsesfest af de Dode ; / cl . og kllire - « le hn . , omtale En . < I ! c > inmomc > rei ' , v. » . ( I ^ eai . ) erindre . Ooinmenhant , e , H . Mand el . <b>Kvinde</b> , som begynder at lcere et Fag ; Begynder . d ! < ) inineneeiiient , m . Begyndelse ; forste Aarsag ; / > l . Underviisning i Begyndelses » gnmdene ; clonner - il c ^ e . , give Anledning til Noget ; prenclre - , pien < lre Bou - , tåge sin Begyndelse ; il v a - 5 tout , Alt maa have en Begyndelse ; leB - 8 Bont tou ^ nui-8 clilncileB , Begyndelsen er altid vanskelig ; lvu - , i Verdens Begyndelse ( brnges kun i Skriften ) . v. « . begynde ; - < ^ n . , give En den forste Underuiisning , lcere ham de forste Begyndelsesgrunde ; v. n . gjore Begyndelsen , tåge sin Begyndelse ; ( kiov . ) a moitie kn.it c ^ ui eommenee , rask begyndt er halvt fnldendt ; n ' a paB luit c ^ ui eommenee , begyndt er ikke fuld > endt ; il eoinmeuee pm ' ou leB autreB KniBBeiit , han begynder der , hvor Andre ende ; - d , begynde en Handling , der skal have Fremgang el . udvikle sig imod et vist <b>Maal</b> : l ' enkant oommenee n , liro ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111103014'target=_>, 1857, Christiania-Postens Feuilleton</a><h4>3558</h4>« Her mig et Oieblik endnu , " vedblev jeg . „ Icg forcstaar at tåge Dem og Laura mcd mig imsrgcn tidlig til Skotland . De maa endelig ikke jamre saaledes ! Jeg taler kun om Neisen mcd et agtestabeligt <b>Formaal</b> . I Skotland , Mrs . Vaggs , er Mgtestnbrt gyldigt , nnar en Mand og en <b>Kvinde</b> i ccl Vidnes Paaher uttale Vielsesformularcu , og en saadan Vielse er , som De tydelig maa kunne indsre , den eneste sittre Utvei for en Brutgom i min Stilling . Hvis De samtykker i at reise med os til Skotland og tjener os som Vidne ved Vielsen , vil jeg med Glade <b>vise</b> Dem min Erkjendtlighcd for Deres Godhet » i Englands Banks veltalende Sprog , som udtryttes for Verden i Almintclighet Paa Overfladen af en Fcmpuudsnote . "

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010092803030'target=_>Luther, Martin, 1855, christelig Postille</a><h4>511</h4>Skriften . Saaledes Icese vi i Evangeliet paa Juledag , hvorlunde Engelen , som forkynder Christi Fodsel , siger til Hyrderne : Og det skal vcere Eder et Tegn : I skulle finde et Barn svobt , liggende i en Krybbe ! Hvorfor giver han dem ikke Maria og Joseph til et Tegn , men ncevner kun Svsbet og Krybben ? Grunden er den : Gud vil ikke <b>vise</b> os til nogen Helgen , ei engang til Herrens <b>Moder</b> , thi de kunne alle feile ; derfor maa Han angive os et sikkert Sted , hvor Christus ligger : det er Krybben . Der sinder man Ham vist , om end Joseph og Maria ikke vare der . Det vil sige saameget , som saa : Christus er indviklet i Skriften , som Legemet i Svobet ; Krybben er Evangelii Prcediken , hvori Han ligger og hvori man tåger Ncering for sit Hjerte . Nu synes det vel rimeligere , at Barnet skulde ligge der , hvor Maria og Joseph , de fromme , hellige Folk , ere ; dog henviser Engelen til Krybben alene , den vil han ikke have foragtet . Evangelium er et ringe og simpelt Ord , dog ligger Chriftus deri . Det Samme lcere vi ogsaa af Fortellingen om den hellige Simeon , der havde den Forjettelse af Gud , at han ikke skulde dse , forend han havde seet Christum . Han kommer i Templet af Aandens Drift ; der sinder han Barnet og tåger det paa sine Arme . Her siges os altsaa , hvor Christus skal findes . Kort og godt , Gud vil advare os mod Mennestelcerdomme , hvor vakkre de end kunne vcere , at vi ei skulle forlade os derpaa , men kun holde os til det eneste og rette Vårtegn , som er Guds Ord . AltOvrigt lade man fare ! Kan vel vcere , at det er ret stuttet og godt sagt , dog vort Hjertes Haab og Fortrsstning ville vi ikke derpaa bygge . Saa lcere vi da af dette Evangelio , at naar den svare Lidelse kommer over os , hvorom vi ovenfor have talet , kunne vi umuligen sinde nogen Trost udcnfor Skriften og Guds Ord . Og derfor har Gud ladet det skrive , at vi stulle drage denne <b>Lcerdom</b> deraf , som St . Paulus siger : Hvadsomhelst er skrevet , det er skrevet os til <b>Lcerdom</b> , paa det vi skulle haveHaab formedelstTaalmodigheden og Skrifternes Trost . ( Rom . 15 , 4. ) Her siger han , at Skriften er trostelig , at d en giver Taalmodighed og Fortrostning . Hvad et Menneste ellers vil troste sig med , det vcere nok saa stort , det er dog Altsammen usikkert , og Hjertet tcrnker stedse : " hvo veed , om det ogsaa er det Rette ! Gid jeg dog var <b>vis</b> derpaa ! " Men naar det bygger paa Guds Ord , saa kan det , uden nogen Tvivl , sige : " det er Guds Ord , det kan ikke lyve , det kan ikke feile , eller stuffe mig . " Det er derfor dm hoicste Kamp , den

<h4>1584</h4>Det er nu den ene Part , den , som giver Aarsag til Vrede og foruretter sin Nceste . Du stal ikke alene ydmyge dig for Gud og bekjende for Ham , at du har gjort Uret , men ogsaa for din Nceste , — kun da kan du vente Forladelse af Gud . Den anden Part derimod , som er forurettet og mener at have gyldig Grund til Vrede , ham advarer Herren ogsaa , at han gjerne stal tilgive og ikke lade sig bede lange . Thi ellers svcever han i stor Fare , siger Herren . Lader du dig lcenge bede , lader du dig ikke finde villig til Forlig , da vil din Modstandcr befale Sagen i Dommerens , i Guds Haand , og sige : " Herre ! jeg har gjort , hvad jeg kunde . Hos Dig sinder jeg Naade , men ikke hos Menneskene . Vclan , jeg befaler det Hele i din Haand . " Griber nu Gud Sagen an , hvorledes mener du vel , din Dom vil vorde ? Han tåger dig , dig , som hverken vil forlade eller forglemme , overantvorder dig til Tjeneren og lader dig kaste i Fcengsel ; og du skal ikke komme ud deraf , forend du betaler den sidste Skjcerv . Det er : der stal ingen Naade vcerc . Med den sammeMaade , Imaale med , skalEder igjen maales " ) . Saaledes krcever Herren af begge Parter , at de stulle vcere barmhjertige , lade Vreden fare og <b>vise</b> sig kjcerligsindcde indbyrdes . Handle vi derimod , da staaer det femte Bud der , og anklager os , at vi ere Manddrabere for Gud . Straffeu vil da heller ikke udeblive . Deune <b>Lcerdom</b> og den dertil svarende Vandel stal ikke forsvinde af Christenheden , men oves flittigen af Alle . Ellers fortorne vi Gud og stade os selv , ei alene paa Liv og Velfcerd , men ogsaa , efter Herrens Trudsel , paa <b>Sjel</b> og Salighed . Gaae vi nu derimod udenfor det Forhold , vi som Christne staae i til hinanden , og betragte de verdslige Embeder og Retter , da gjcelder ikke her det femte Bud ( uden forsaavidt det forbyder personligt Had og al uretfcrrdig Fremgangsmaade mod Andre ) . Thi her har Gud givet en scrregen Befaling , som man stal folge og efterkomme . Hvo som nemlig beklceder et verdsligt Dvrigheds-Embede , han har Befaling , at han stal vredes , straffe , ja ihjelstaae , naar Undersaatterne begaae nogen ond og Dods-Straf vcerdig Gjerning . Ligeledes har Gud givet Fader og <b>Moder</b> en synderlig Befaling over Born og Tyende , at de ikke stulle see igjennem Fingre med deres Overtredelser , men irettesette - og straffe dem derfor . Det

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100803008'target=_>Overskou, Th., 1850, Comedier</a><h4>2793</h4>< < ^ et er da fcerdigt ; men mat og koldt ! Aldrig har jeg talt med hende — og dog elsker jeg hende — jeg elsker hende saa hsit , at jeg aldrig kan trccffe bendes Billede saalcdcs , som det sva ' ver for min <b>Sjel</b> . Emedesvendcn Qventin Messis blev Maler for at male sin <b>Piges</b> Portrait ; men var hans Kjccrlighed saa stcrrk , at den gjorde ham til Kunstner , saa kunde hans Kjcrrlighcd ikte verre tilfreds med hans Kunstvcerk ; thl den Elskede har altid Mdighcder , som de mcest glodende Farver ikke kunne male . lgaaer hen oc , ster ud af Vinduet . ) Hun er der ikke ! O hun tthner ikke , at jeg har sogt at fremlokke de Dndigheder paa Lccrredct , som Naturen saa rigeligt har skjcrnket hende . — Aldrig har jeg mindre naaet mit <b>Maal</b> , og dog er dette Villede mig det dyrebareste jeg eier .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010012603001'target=_>Wexels, W.A., 1837, Forklaring over Det nye Testamentes Skrifter</a><h4>887</h4>V . 28. Kjcerlig , som en <b>Moder</b> sine vansmcegtende Born , kalder Frelseren til sit Hjerte alle dem , som " arbeide " , som kcempe og strcebe efter at vorde Synden los , alle dem , som sukke under dens Aag , trykkes af dens Byrde , fole Lovens Forbandelse og mcegte hverken at boere den eller at afkaste den , vorde maastee dertilmed mere og mere cengstede og betyngede ved menneskelig <b>Lcerdom</b> , ligesom Folket paa Jesu Tider blev det ved Pharisceernes og de Skriftkloges Lovbud og Forskrifter , Lue . 11 , 46. Ap . Gjem . 15 , 10. Coll . 2 , 20. Gal . 4 , 3. 9. 24. 26. o . st. St . Saadanne indbyder Jesus , at de med deres Sorg og Byrde stulle komme til ham , saa vil han tåge Byrden fra dem , stjcenke dem Fred ved Syndernes Forladelse og en frivillig Kjcerligheds <b>Aand</b> , der uddriver den Frygt , som haver Pine .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081903006'target=_>Kurtz, Joh. Heinr., 1859, Lærebog i kirkehistorien for studerende</a><h4>959</h4>4. Den alerandrinste Skole . Classist Daunelse , philosophist <b>Aand</b> , dristig Frisindethed og skaberisk Kraft udmerkede de fleste af dens Lerere , der fornemmelig havde stillet sig den Opgave , at stille en sund kirkelig Gnosis ligeoverfor den falske , heretiske Gnosis , hvorved rigtignok selv dens beremteste Lerere ikke bleve frie for Speculcttionens uevangelifke Vildfarelser . Pleiestedet for denne theologiste Retning var fornemmelig Kate chets koten i Alexandrien , der fra at vere en Anstalt for dannede Katechumenels Underviisning var opvoxet til at blive et theologist Seminarium . Deune Anstalts ferste beremte Lerer var Pantenus ( - f - 202 ) . Han overstraaledes af sin Discipel og Efterfelger Clemens Alex . Paa sine Neifer , som han foretog af Videbegjerlighed , tom han til Alexandrien som Hedning , allerede udrustet med omfattende Lerdom, og her fengsledes han af Pantenus , der omvendte ham . Medens Forsagelsen vårede under Septimius Severus ( 202 ) , unddrog han sig ifelge Matth . 10 , 23 ved Flugten for Hedningenes truende Hevn . Men han blev utrettelig ved at virke for Kirken indtil sin Ded 220. ( Bedste Udg . af 1. putter , oxon . I ? ts . 2 Voll . loi . ) Saa stor endogsaa hans Beremmelse var , overstraaledes han alligevel af sin Discipel og Efterfelger Origenes , et Vidunder af <b>Lerdom</b> , der belragtedes med Forundring af hedenste og christelige Samtidige , ogsaa kaldet og X « > ixl > it ^ » < i ? paa Grund af sin lernftid , udmerket som Philosoph , Philolog , Kritiker , Exeget , Dogmatiker , Apologet , Polemiker etc. , hvem Efterverdenen med lige Net deels erede som den egentlige Begruuder af tirkelig-videnfkabelig Theologie , deels reproberede som Ophavsmand til mange heretiske Anskuelser . Han var fedt A . 185 i Alexandrien af christelige Foreldre , blev dannet af sin Fader Leonidas , ligesom ogsaa af Pantenus og Clemens , indgjed , endnn nesten Dreng , sin Fader Mod i hans Martyrded under Septimius Severus ( 202 ) , blev Forserger for stu hjelpelefe <b>Moder</b> og sine sex faderlese Sedstende og kaldet as Biskop Demetrius til Lerer ved Katechetstolen ( 203 ) . For saavidt muligt at kunne opfylde sit nve Kald , kastede han sig nu med Iver over Philosophiens Studium ( under Neoplatonikeren Ammonins Sakkas ' s Veiledning ) . I sit Levnet var han heist tarvelig og fra Ungdommen aj streng Asket . I sin Streben efter christelig Fuldtommenhed ( paa Grnnd af det misforstaaede Sted Matth . 19 , 12 ) castrerede han sig , men cmertjendte selv fenere det Forkastelige ved dette Stridt . Hans Beremmelse voxede imidlertid Dag for Dag . Idet han fulgte en heist erefuld Opfordring , virkede han en Tidlang for Missionen i Arabien , blev af den fortreffelige Julia Mammeci kaldet til Antiochien ( 218 ) og foretog A . 228 i kirkelige Anliggender en Reise til Palestina , hvor Bistopperne af Cesarea og Jerusalem indviede ham ( rigtignok imod Kirkelovene ) til Presbyter . Hans Biskop Demetrius , desuden iversyg over Origenes ' s dagligt voxende Beremmelse , felte sig derved krenket i sin biskoppelige Anseelse , kaldte ham tilbage og lod ham paa to aler . Synoder ( 231 og 232 ) afsette fra sit Lereembede og excommunicere paa Grund af Vrcmglere , Selvlemlestelse og Foragt for Kirkelovene. Origenes begav sig nu til Cesarea og aabnede der , eret og hedret og understettet af Philip Arabs , en theologist Skole . Hans literere Virksomhed naaede her sit Toppunkt . Men under Decius blev han tågen tilfange og dede i Tyrus forn en Felge af de grusomme Pinsler , han standhaftig havde lidt ( 254 ) .

<h4>1120</h4>I . Cuseoius Pamphili , Bisl . i Ccesarea i Palcestina 5 338. Til Confessoren Pa mp hil us , som han kaldte sin Fader , stod han i det inderligste Vcnstabsfoihold og deelte med ham Begeistringen for Origenes ( § 63 , 2 ) . Han befad ogsaa i hei Grad Keiser Konstantin den Stores Tillid , der aabnede ham alle Rigets Archiver for hans historiske Studier . Hans <b>Lcerdom</b> var meget omfattende , haus Flid forn Samler beundringsvcrrdig og utrcettelig , derimod mangler han Acmdsdybde , fpeculativ <b>Begavelse</b> og fnldendt dogmatisk <b>Uddannelse</b> . Men med desto stsrre Tak maa man rose Grundigheden , Omhyggeligheden og Mgtheden i hans omfattende og msisommelige Forskninger . — 2. Den meest fremragende Skikkelse i det 4. Aarh . ' s Kirkehistorie er Athanasius , af den beundrende Efterverden kaldet pater ortttottoxi ae . Hver Tomme paa ham var en Kilkesader , og hans Livs Historie er hans Tids Kirkehistorie ( § 80 ) . Deri udfolder sig et Liv , fnldt af Heltemod i Kampen , fnldt af Trostab , Kraft og Viisdom i Opbyggelsen , stort i Undergangen , stort i Seiren ; et Liv , hvori Indsigt , Villie og Handling , Alvor , Kraft og Mildhed , Videnstab og Tro staae i stjsnneste Samklang med hverandre . I . A . 319 blev han Diakon i Alexandrien. Hans Biskop Alexander erkjendte snart den unge Mands eminente <b>Begavelse</b> og tog ham med paa Conciliet i Nicoea ( 325 ) , hvor han begyndte sit Livs Kamp . Snart derefter dsde Alexander , og Athanasius blev hans Efterfslger ( 328 ) . I hele 45 Aar var han Bistop , men blev ti Gange fordreven og tilbragte 20 Aar i Landstygtighed , meest i Vesterland ( f 373 ) . Hans Virksomhed som Skribent er for stsrste Delen helliget Kampen imod Arianismen ( Mcmiinerudg . af Uontlaiieon . par . 1698. 3 Voii . ) . — 3. Basilius d . Store , Biskop i sin Fcedrenestad Ccesarea i Kavpadocien , et i Sandhed „ kongeligt " Phcenomen i Kirkehistorien , - f - 379. Hans <b>Moder</b> Em me lia og hans Bedslemoder Makcina plantede alt tidligt et fromt Sind i Drengens Bryst . Studerende i Athen sluttede han en Venstabspagt for Livet , grundet paa Kjceilighed til Kirken og Videnskaben , med sin ligesindede Landsmcmd Gregor fra Nazianz , hvortil senere bans Broder Gregor , Biskop i Nyssa , sluttede sig med samme Inderlighed . Lang Tid levede han som Asket i Ensomheden , bortgav hele sin Formue til de Fattige og blev derpaa Presbyter og Biskop . Mandens hele rige Liv acmder verdensovervindende Tro , selvfornegtende Kjcerlighed , cedelt Sindelag og kongelig Vcerdighed . Med sin Aands Kraft oprelholdt han den katholste Kirke i Orienten under Arianeren Valen s ' s rasende Forfslgelser . Det stjsnneste Vidnesbyrd om hans oedle Sind var den storartede Basiliasstistelse , et uhyre Hospital i Ccesarea , hvortil han anvendte sine rige Indtomster , idet han selv levede i trange Knar . Ogsaa ved sine Skrifter hsrer Basilius til de betydeligste Kirtefcedre . Hans 365 Breve ere et tro Speil af ham felv og hans meget bevcegede Tid . Smlgn . W . Klo se , Bas . d . Gr . nach Leben u . Lehre . Stråls . 1835 ; Bohringer , Kirchengesch . in Biographien I , 2 ( Maurinerudg . af Oarnier og p. Maranu3 . par . 1721. 3 Voll . tol . — 4. Gregor fra Nazianz , fsdt i Arianzus A . 328. Hans Fader Gregorius , tidligere Hypsistarier , men omvendt af sin fromme Mgtefcelle Nonna , var Bistop i Nazianz . Sonnen blev hans Medhjcelper og imod sin Villie hans Efcerfslger , flygtede bort fra dette Embede til Ensomheden , blev Bistop ved den lille , foragtede Niccenermenighed i Konstantinopel ( ved Anastasiatirten , hvor han holdt sine bersmte Taler om Chiisti Guddom , der forskaffede ham Tilnavnet 6 s-ta ) . oi / < , 5 ) , udncevnt af Theodosius d . St . 3 so til Residentseus Patriarch . men vendte allerede i det fslgende Aar , dreven af sine Fjenders Misundelse , tilbage til Nazianz og f 391. Smlgn . C . Ullmann

<h4>1123</h4>Greg . v. Naz . d . Theol . Darmst . 1825 og Bo bringer I , 2 ( bedste Udg . af v. ( ttemeneet . par . 1778. 2 Voll . tol . ) . — 5. Gregor fra Nyssa , Basilius ' s yngre Broder . I philosophist <b>Begavelse</b> og fuldendt videnskabelig <b>Dannelse</b> overgik han de to celdre Venner . Hans theologiske Ansknelse har ogsaa langt dybere Nod i Origenes end deres . I Iver for at bekcempe Arianismen stod han itte tilbage for dem , og hans BelSmmelse hos Samtid og Eftelvelden er neppe ringere end deres . Smlgn . I . Nupp , Greg . v. Nyssa , Leben und Meinnngen . Lpz . 1834 ( bedste Udg . af jVroitton le Due . par . 1615. 2 Voii . tol . ) . — ' 6. Didymus d . Blinde mistede alt Synet som fireaarigt Barn , men erhvervede sig alligevel en beundringsverdig <b>Lcerdom</b> . Han var Katheket i Alexandrien og dsde A . 395. Hans Skrifter vare talrige , men kun meget faa ere bedårede . Som en begeistret Beundrer af Origenes deelte han mange af hans cxcentriske Anfknelser ( Apokcttasiasis ) ; men ogfaa han har ifslge denne Tids kirkelige Kampe givet sin Theologie en afgjort kirkelig Vending . — 7. Synesius , Bisk . i Ptolemais i lEgypten , var en Discipel af den bersmte Hypatia ( § 66 ) og begeistret Beundrer af Plato , f 430. Som lykkelig HEgtefelle og Fader og velhavende , levende for Studiet af Philosophien . kunde han ikke beslutte sig til at modtage Bispedsmmet . Aabenlyst tilstod han sin origenistlste Heterodoxie angaaende Opstandelsesleren , og ligesaa aabent erklcrrede han , at han ogfaa forn Biskop vilde fortsette sin egtestabelige Forbindelse med sin <b>Hustru</b> , og Ingen tog Forargelse deraf . I det biskoppelige Embede udmcrrkede han sig ved « del Iver og et Mod . der ikke kjendle nogen Menneslefrygt. Han var Hvmnedigter og Forfatter til philosophiste Afhandlinger ( Udg . af Petaviuß . par . 1612. tol . ) . — 8. Epiphanius , Bifkop i Salcimis paa Cypern . nedstammede fra Palestina fra jsdiste Forceldre , en Elev af den h . Hil ari on og de sketiske Munke ( - f - 403 ) . I sin biskoppelige Virksomhed var han et Msnster paa Trostab og Iver , navnlig ogsaa utrettelig opoffrende og felvfornegtende i at pleie de Fattige . Men i Forgrunden for al hans Tenken og Virken stod hans Iver for kirkelig Orthodoxie . Han var en grundccrlig , sandhedstjcerlig , godmodig Mand af noget indstrenket <b>Aand</b> og meget ensidig Dannelfe , uden al Knndstab til Verden og Menneskene , ude af Stand til at overskue Forholdene , noget egensindig og meget lettroende , men yderst lcerd , flittig og ikke nden Forstand og Skarpsinoighed . Hans munkeagtige Opdragelfe havde indplantet ham en uudstnkkelig Afsky for alte Vranglerere , og fra den ftetiste Orken medbragte han den urokkelige Overbeviisning , at Origenes var den oprindelige Fader til de arianske og alle svrige Kjetterier . Smign . § 81 ( Udg . af v. petaviuL . par . 1622. 2. Voll . loi . ) . — 9. Kyrillos , Patr . i Alex . , Neven , Elev og Efterfslger af Theophilus . Ontlens zelotiste og voldsomme Adferd blev itte uden Indflydelse paa Nevenens Charatteerdannelse . Om hans Liv og Kampe smlgn . § 82 , 3 ( Udg . af h . Skr . af 5. H.uuertuß . par . 1638. 6 Voll . loi . ) .

<h4>1978</h4>Stifteren af Dominikanerordenen var Dominicus Gutzman , fodt 1170 af fornem castiliansk Slegt , Prest i Osma , en Mand af besindig <b>Aand</b> og lerd <b>Dannelse</b> . Hans glodende Iver for Menneskenes Sjelefrelse drev ham med nogle Medhjelpere til det sydlige Frankrig ( 1208 ) , for i apostolisk Fattigdom og Selvfornegtelse at arbeide paa Albigensernes Omvendelse . I A . 1215 gjorde han en Pilegrimsreise til Rom . Innocents 111. gav ham en Regel , som Honorius 111. udvidede . Dominikaner- eller Prrrdikcordenen ( circlc » trittrum pr » 6 cliolttoruln ) fik derved det Privilegium , at maatte predike og modtage Skriftemaal allevegne , med den specielle Opgave , ved Prediken og Lere at fore Kjetterne tilbage til den encsaliggjorende Kirke . Forst fenere ( 1220 ) erklerede Dominicus , idet hau efterlignede Franciskanerregelen , sig og sin Orden for Tiggere og dode 1221 med en Forbandelse over hver Den , der vilde besmitte hans Orden med Besiddelsen af jordisk Gods . Han blev ogsaa canoniseret af Gregor IX . Af omvendte Albigenfere havde der dannet sig en <b>kvindelig</b> Forgrening af Ordenen , der ogsaa senere sluttede sig til Tertiarerne ( trl > tr6B 6 t 80 lor68 cl 6 miiiticr ( UrriBti ) .

<h4>2038</h4>I . Striden . — Petrus Abcrlard , der langt overgik sine Samtidige i Skarpsindighet » , <b>Lerdom</b> , dialetlist Dyglighed og dristig Frisiudethed , men ogsaa i Overmod og Disputeersyge , var fodt 1079 i Palais i Bretagne . Hans Lerer i Philosophie var Vilhelm fra Champeanr i Paris , sin Tids meest ansete Dicilektikcr . Men snart overvandt Disciplen Mesteren i offentlig Disputcits og stiftede nu Skolen i Melun ved Paris , hvor Tilsender af Disciftle forsamlede sig omkring ham . For at vere nermere ved Paris , forlagde han sin Skole til Corbeil , kort efter umiddelbart foran Murene at Paris paa St . Genovevabjerg og ophorte ikke med at udfordre og ydmyge Vilhelm , for denne rommede ham Marken . For at bane sig Vei til en endnu mere glimrende Berommelse begyndte han at studere Theologie under Vcholastikeren Anselmus fra Laon . Men meget snart trode den overmodige Discipel ogsaa at kunne oversee denne Lerer . Han gik tilbage til Paris , hvor der atter samlede sig et stort Antal enthusiastiske Disciple omkring ham . Kanuiken Fulbert valgte ham til Lerer for sin Niece Heloise , der udmerkede fig saavel ved Skjonhed som ved <b>Aand</b> og <b>Lerdom</b> . Han vandt hendes Kjerlighed , men hun gav Afkald Paa at kaldes hans <b>Hustru</b> , for ilte at lukke den elskede Mand Veien til de hoieste 2Eresposter, og lod sig hemmelig vie til ham . Men da Heloise haardncikket neglede Mgtestcibet for Verden , og derfor behandledes haardt af sine Slegtninge , bortforte han sin frugtsommelige <b>Hustru</b> til Nonneklostret i Argcntenil . Hevntorstende lod Fulbert ham derfor overfalde om Natten og castrere . Fuld af Skam og Fortvivlelse flygtede han til Klostret St . Denys og aflagte her Muukeloftel . Heloise tog Sloret i Argenteuil . Men Abelard maatte ogsaa her i St . Denys give efter for sine tidligere Disciples stormende Bonner og holde Forelesninger . Hans letferdige Behandling af Kirkeleren og hans hovmodige Spot forskaffede ham megtige Modstandere , der nodte ham paa Synoden i Soissons i Nerverelse af en pavelig Legat ( 1121 ) til at taste sin just udtomne Lerebog i Theologien ( Intrncluetic » in tbeolnzwm ) paa Ilden og domte ham til Klosterfengsel. Ved Hjelp af nogle Venner blev han dog snart fri igjen og vendte tilbage til St . Denys . Men da han gjorde den Opdagelse , at Dionysius fra Paris ikke var Areovagiten , nodte Muuteues Forfolgelse ham til at flygte ind i en Skov ved Troyes . Ogsaa herheu forfulgte hans Disciple ham og tvang ham til at holde Forelesninger . Under deres Hceuder voxede hans Eremitbolig op til det ansete Abbedie Par akl et. Da han heller ilke her fik nogen Ro , overgav han Paraklet til Heloise , der som Abbedisfe i Argenteuil ikke havde kunnet komme ud af det med stue uordentlige Nonner , — blev Abbed i et Kloster i Bretagne og optraadle atter med uhyre Bifald forn Lerer i St . Genoveva, efterat han i hele otte Aar forgjeves havde bestrebt sig for at gjenoprette Klostertugten . Hau strev en Ethit under Titelen : Beitci to ivßum > udgav paany sin Intrncluctic » under Titeleu : Ill6alc » Big6 6 llrißtillNll6 1,1 . V . og stillede , for at hanne Traditionstheologien , i Bogen Bio 6 t non en Mengde hinanden modsigende Steder hos Kirkefedrene ved Siden af hverandre . Hans Forelesninger vakte nhyre Sensation . Nu optraadte den hellige Bernhard

<h4>3016</h4>2. lansenismen i dens forste Stadium . ( Smlgn . iVleloilior it,o)--clooli.sr, clo nißt . lanßonißmi 1,1 . VI . Iraj . licl Nli . 1695. ( < 3. Oornoron ) Uißt . geueraio < 1 u lanßenißmo . 171 l . 3 Voll . — H . Reuchlin , Gesch . v. PoNroyal . 1839 , 1844. 2 B . ; Bainte.Louve . ? ort-roM . 1840. 2 V . ; 6 ro2ioiro , 103 ruineß cio port-rciM . par 1809 ; Reuchlin , Pascal ' s Leben . Stuttg . 1840. ) — Biskop Cornelius Jansen af Ypern ( - f - 1638 ) havde indviet hele sit Liv til et omhyggeligt Studium af den hellige Augustins Skrifter . Frugten af dette Studium var et lerd Verk med Titelen H.uFlißtiliuß , der ferst blev udgivet efter hans Ded ( 3 B . Fol . ) Da den store Kirkefaders Lere om Synd og Naade her var udviklet i hele sin Sandhed og Skarphed . angreb lesuiterne heftig Bogen og udvirkede hos Paven et Forbud mod samme ( 1642 ) . Men Augustins Lere havde ogfaa i Frankrig mange ved <b>Aand</b> og <b>Lerdom</b> udmerkede Venner . Dertil herte den fortreffelige Jean Duvergier de Haurann e , Abbed i Benedictinntlostret St . Cyran , - j - 1643 , og den itte mindre dygtige Lerer ved Sorbonneu , Auton Arnanld . Denne Sidste kom fnart ved sine Skrifter ( Do ili lreguonto eomniunion imod opuß nperatmn i Sacramentet ; I , a tlieoloFiL morals ciLB le3uiteß ; ln morals rn-aeticilio cl6B i aabenbar Strid med lefuiterne . De bevegede Innocents X . til at fordemme feM jansenistiske Setninger forn kjetterske ( 1653 ) . Tilhengerne af det augustinste Lcnebegreb lode vel den pavelige Afgjerelfe stace nantastet , men paastode , at de ikke stode i Icmsens i den Betydning, hvori Paven havde fordemt dem . Paa lesuiternes Opfordring blev nn Arnanld udstedt af Sorbonuen . Han fcmdt Tilflugt hos sin Sester An gelica Arnanld , Abbedisse for Cistercienser-Nonneklostret i Por tro y al ved Paris , en <b>Dame</b> af dyb-alvorlig Neligiesitet . Portroyal blev ved ham det religiese Livs og den religiese Slrebens Midtpunkt for Frankrig . Nesten paa de gamle Anachorelers Viis samlede der stg omkring dette Kloster et Antal af Frankrigs acmdrigste og frommeste Mend , Augustins famtlige Beundrere og Fjender af lefuiternes fordervelige Moral . En Aandsbestegtet af dette edle Broderstad var den dybsindige og aandrige Mathcmatiker Blaise Pascal ( Forfatter af de dyde ? siiß66B Bur ia reliFinn ) . Under Navnet Louis de Mo utalte udgav han 1656 stue beremte I ^ Lttroß provinLialeß , hvori han med authentiste Bilag og med ligesaa dyb Alvor som fim Vittighed blottede mange lefuiters fordervelige Moralgrundsetninger i hele deres Afstyelighed . Bogen gjorde nhyre Opsigt ; men lesuiterne hevnede sig ved en pavelig Bulle ( 1656 ) , hvori det blev fastholdt , at Jansen havde lert de fem omhandlede Setninger just i

<h4>3487</h4>forogedes Aar for Aar . Der existerer nu i den protestantiske Verden 34 store Hovedselstaber med utallige Filialforeninger , der aarlig anvende henimod 8 Millioner paa Missionen og underholde paa 1400 Misstonsstationer henimod 3000 europeiske og amerikanske Missionerer og ligesaa mange nationale Medhjelpere. England indtager endnu den forste Plads i Missionsbidragene , nestefter det Nordamerika og Tydstland . Bredremenighedeil hevder endnu stedfe sin gamle paa fin <b>Vis</b> uopnacede Berommelse . Iblandt de nyopstacede Hovedforeninger ( med men eller mindre talrige Biforeninger ) udmerke folgende stg indenfor den reformeerte Kirke : Det store amerikanfke Misfionsselfkab i Boston siden 1810 ( Lonrcl ol loreizn inißßionß ) og den amerikanske Baptistmission siden 1814. Desforuten har Nordamerika et methodistisk og et biskoppeligt Missionsselfkab af Betydenhed . De fleste i Tydstland opstaaede Foreninger hore principielt til den unerede Kirke . De vigtigste el-e det i Bafel siden 1816 , i Berlin siden 1823 ; det rhinste med Missionsseminaret i Barmen siden i 829 ( der har en mere confodercttiv Charakteer , med overveiende lutherske Elementer ) — det nordtydske , siden 1836 med deres Sendebuds Forptigtelse paa den angsburgste Confession , men med Utelukkelse af de ovrige lutherske Bekjendelsesstrifter . En tempereret luthersk Charakteer hevdede Missionsskolen i Berlin , der stiftedes af Icinicke ( siden 1800 ) ; i dens Fodspor gik den Gostnerske Missionsforening i Berlin . En bestemt luthersk Charatteer antog Missionsselskabet i Dresden siden 1836 , hvis Seminarium 1848 forlagdes til Leipzig , foråt dets Elever kunde drage Nytte af Universitetet . Det har gjenoptaget det gamle lutherske Missionsverk i Ostindien ( § 197 , 7 ) . I Sverrig , Danmark , Norge . Ostersoprovindsnne , Baiern , Hannover , Mecklenburg , Hessen og Nordamerika er der ligeledes opstacet udelukkende lutherske Foreninger , deels selvstendige , deels med Tilslutning til Dresden-Leipzig . Ca s sel har serlig Oiet henvendt paa China . Med serdeles Roes maa man ogsaa fremheve Herman nsburger Anstalten i Hannover under Bestvrelse af den valtre Pastor C . Harms , der 1853 har udsendt et eget Missionsfkib , for at stifte en Mission blandt Gallcerne i Afrika . — Det er ogsaa en udmerket Fortjeneste af den evangeliske Mission , at have bevirket Slavehandelens Ophevelse af de europeiske Stormager ( 1830 ) , og alle Slavers Befrielse i de engelske Colonier ( siden 1834 ) , hvorfor det engelske Folk offrede 160 Millioner . Den edle Wilberforce ( - st 1833 ) har med utrettelig Udholdenhed offret sit hele Liv for dette <b>Maal</b> . — For Missionen iblandt loderne dannede der fig ogsaa nye Foreninger i England , Tydstland og Frankrig med opoffrende Virksomhed , dog uden tilsvarende Held . Begynte vi Orienteringen Paa den protestantiske Missions Gebeet med det nordlige Europa , saa mode vi forst deu fvenste Mission i Lavland , der kraftigen trivedes , fornyet siden 1825 af den fortreffelige Stockfleth ( § 189 , 0 ) . I det nordlige Amerika <b>moder</b> os Herrnhnternes velsignede Mission i Grenland og Labrador . Iblandt Nordameritas oprindelige Beboere og Vestindiens Negerslaver virkede herrnhutiste , methodististe , baptistiske og engelsk - biskoppelige Missionerer stadigt med Held og stor Iver . I Sydamerika har Brodremenigheden sat sig fast paa Mosquitokysten og i Paramaribo. — Paa Afrikas Vestkyst grundlagdes fra England af Sierra- Leonecolonien til befriede Negerstavers Bosettelfe og Christianisering , fremdeles imod Syd fra Amerika af i samme Hensigt Colonien Liberia . Begge stane i en blomstrende Tilstand under methodistisk , baptistisk og engelsk . biskoppeligt Tilsyn . Paa G uld kyst en har Evangeliet vundet Indgang ved Baseler Selskabet , i Gammel - Calabar ved Baptisterne , omkring Gabunstoden ved det amerikanfke og det nordtydske Selstad . For Sydafrika danner Kapstaden Udgangspunttet for den christelige Civilisation . Serdeles velsignelfeslig var her Herrnhnternes Missionsvirtsomhed iblandt Hottentotterne , Vniinermissionen dirkede blandt de vilde Koranncer og det evangelisk-franske Selstad blandt Betschuanerne.

<h4>3637</h4>behendigt al lavere mellem Scylla og Charybdis . For at opnace det Forste , underhandlede de med Demokratiet , for at undgaae det Sidste , med Reactionen , og det lyttedes dem hos Begge at afvinde Fordele . I Wurzburg forsamlede de tydske Bistopper sig ( i November 1848 ) til felles Raadstagning om den fordeelagtigste Fremgangsmaade under disse kritiske Omstendigheter . Urokkelig Trostab imod Nom var det Forste , de besluttede sig til , beredvillig ! Samtykke til Fedrelandets « politiske Gjenfodelse " det Andet , talnemmelig Modtagelse af Tilsagnet om übetinget Samvittighetsfrihet » ( i Frankfurter Parlamentets Grundrettigheder) for at tilveiebringe Kirkens abfolute Selvstendighet » og gjennemgribende Bestyrelse af Folteopdragelfen ligefra Elemeutarstolen indtil Hoiskolen var det Tredie . Et vigtigt Middel til at heve Klerus og gjenoprette Kirketugten saae de i Gjenoprettelsen af Diocesansynodnne , men den hell . Fader ansane dog Midlet endnn forelobig for altfor betenteligt . Men kort storveien var der blevet oprettet en Institution af udstrakt Betydning , hvortil ogsaa Legfolk — og disfe fortrinsvis skulde medvirke , nemlig Piusforeningen , et katholst Sidestykke til den Potestcmtiste Kirkedag . Lige efter Mattsdagene var der paa flere Steder i Tydstland opstacet Foreninger til at pleie og forsvare de katholste Interesser . Paa den neste Domkirkcfest i Coln ( Aug . 1848 ) samledes flere af disses Medlemmer og aftalte en Generalforsamling i Oct . 1848 i Mainz , hvor den forste Forening af denne Slags var opstacet under Navn af Piusforeningen . Her sluttede alle eutelte Foreninger stg sammen til eu stor Fellesforening under Navnet : „ Tydftlands katholfke Forening " ; dog foretrak man i Praxis det kortere Navn : Piusforening . Til at bestyre Fellesforeningens Forretninger skulde der hver Gaug af Generalforsamlingen , der fornyedes hvert Aar , udvelges een af Specialforeningerne , der forer Navnet « Forsted " ( Vorort ) . Som Formant bestemtes Folgende : Kamp og Forsvar for Kirkens Frihed og dens Herredomme over Skolen ; Folkedannelse i katholst <b>Aand</b> og Udovelse af christelig Barmhjertighet ) ; som Grundlov Lydighed imod Paven og Episkopatet , fredelig Stilling til Staten og enhver bestaaende Statsforfatning, forsaavidt Kirkens Rettigheder itte derved krenkes , og defensiv itte agressiv stilling til de ilke-katholske Confessionen Gnds <b>Moder</b> blev udkaaren til Foreningens Patronesse . Ethvert Medlem forpligter sig til et Fadervor og et Ave-Maria hver Dag til Foreningens Bedste . Paa den anden Forsamling i Breslan forelagtes der en Skrivelse fra Paven med apostolisk Billigelse og Velsignelsesonske . Forsamlingen i Wien ( 1853 ) maatte dog indromme , at det endnu havde lange Udsigter med den forhaabede Tiltrekning af Folkemassen , da allid kun de samme Ansigter viste sig . Forsamlingen i Coln 1854 maatte bortfalde , da Comiteen vegrede sig ved at give den preussiske Regjering Garantie for , at politiske Expectorationer og confessionel Polemik holdtes borte . De enkelte Foreninger have mangfoldige <b>Formaal</b> for Oie . Bonifaciusforenin gerne f . Ex . have til Hensigt at understotte trengende katholste Menigheder i det protest . Tydstland ( altsaa Efterligning af den prot . Gustav-Adolfsforening ) , Karl-Vorromeusforeningerne Udbledelsen afgote katholste Stnfter , Vincentius og Elifabethforeningerne Sygebesog og Fctttigpleie , Sveudeforeuingerne ( stiftede allerede 1846 af den valtre Oldermaud K o l p i n g i Elberfeld) Haandvertsfvendenes aandelige og legemtige Pleie ; Jesu hell . Barndoms Foreninger bestane forstorstedelen af Born , hvoraf ethvert maanedlig bidrager 5 Peilninge for udscttte hedenske Vorns Redning ( iser i China ) og daglig beder et Ave-Maria for dem . Foreningen har ogsaa bemegtiget sig den politiske Dagspresse og stiftet en af yr . Brischar i Wien redigeret alm . „ katholst Literaturtidende " 1855. Desforuten er der Tale om Stiftelsen af et katholst Universitet i Tydstland , om Kuustsoreninger o . s . v. For den katholste Mission iblandt Hedningene og de Vantroe bestacer der i Nom foruden Propagandaen endnu 14 andre Dannelsesanstalter ( det tydst-ungarfte , engelste , stotste , irlandste , greske o . s . v. Collegium ) , i Paris tre ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2007062210023'target=_>, 1862, Nat og Morgen</a><h4>405</h4>siaae , for ikke at blive nsdt til at drikke Våndet , men han negtede standhaftig at lyve . Da alle Anmodninger vare forgjeves , raadede man ham til at drikke det rsde Vand paa et afsides liggende Sted , paadet at han ikke skulde blive fon hexet afFienderne .  Nei , " sagde han , . , her vil jeg drikke det for mine Fienders Oine , og naar Gud er med mig , kan ingen Troldkarl og ingen Djcevel fordcerve det rsde Vand . Saa blev da Dagen bestemt , og i den Tid , der hengik her imellem , blev han meget overhcengt af sine Slcegtninge og sine Anklagere om ikke at drikke , men heller tilsiaae . Dagen kom . Han blev fort hen paa Pladsen , klcedt nsgen og omgjerdet med Platanblade . Om Morgenen havde han faaet omtrent to Thesteer Niis ; kastede han disse op tilligemed det rsde Vand , saa var han uskyldig . D sch em besieg det dertil opbyggede Stillads og dråk otte Kalabesker , efter vort <b>Maal</b> to rigelige Kander , rsdt Vand , som man heldte i ham saa hurtiyt , som han kunde svcelge det . Efter nogle Minuters Forlsb opkasiede han det altsammen tilligemed Nisen ; men fsrend han kunde forlade Stilladset , blev han afmcegtig . Da hedte det , at han vel var uskyldig , men der maatte dog vcere et Hexepalaver ( Forsamling af Here ) i hans Bug , siden Djcevelen ncesien gjorde det af med ham .  Nei , " sagde han , da han kort efter kom sig igjen ,  jeg dråk kun for at fsie Eder og for at <b>vise</b> Eder , at jeg ingen Troldkarl er ; men dersom I endnu holde mig for skyldig , saa bringer mig hen til eders Afgudsbillede Sengha , at han maa drcrbe mig , hvis han kan . " Dermed var Sagen afgjort . Efter nogle Dages Forlsb komDschem atter paa Arbeide , men han var ikke mere saa tilgjengelig for Sandheden , som fsr . Nogen Tid efter fiygtede ogsaa Dschems <b>Kone</b> , som man nu anklagede for Trolddom , ind i Missionshuset , og da Missioncer Nylcrnder sirar derpaa reiste til Sierra Leone , saa tog han D sch em med sig som Roerskarl tilligemed <b>Konen</b> og lod dem blive der . Uden dette vilde Begge ufeilbarlig have virret fortabte , medens det christelige Liv i Kolonien maatte have den meest velgjsrende Indfiydelse paa dem Begge .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014090108027'target=_>Rosenius, C.O., 1868, Hemmeligheder i Lov og Evangelium</a><h4>162</h4>Voerk I Herskere over hele Jorden ! skulde I have et Baand poa Eder ? Skulde I ikke have 11 l Frihed ? osv. Mm da Fristerens Dristighed var voxet , siger han udtrykkeligt : I skulle blive ligesom Gud eller „ Guder " , og just dette : I skulle blive ligesom Gud , nemlig ikke ved nogen Guds Naade og Gave , men ved egen „ <b>Kundskab</b> om Godt og Ondt " , ved eget Foretagende , egen Gjerning Mcrrker hos Adams Slcegt , sum endnu idag uophorligt stikte frem . Uncgtelig er det ogsaa noget Betydningsfuldt , at Slangen opvakte et Kundstabsbegjler , som i Særdeleshed bidrog til dette store , forskrækkelige Fald . Lnther siger : „ Det er egentlig Djævelens naturlige Gift , at Mennesket vil voere klogere , end det er ham befalet af Gud " . Forst er der i Almindelighed Intet , der har gjort Menneskene saa daareagtigc i aandel ' ige Ting , saa mcxgtigt fjendtlige imod Gud og hans Veie som den indbildske Leerdhed . „ Ter de holdt sig for <b>vise</b> , bleve de Daarer " , ogsaa om de virkelig besad det stonte Forraad af <b>Kundskaber</b> . Apostelen siger at „ <b>Kundskaben</b> oMwser " ; en storre Rigdom paa Forstand og Viden bliver derved den mægtigste Fristelse til Selvforgudelse og Selvklogskab , og blot derved er det Menneske mere end noget andet übekvem for Guds <b>Visdom</b> . „ Gud staaer de Hoffoerdige imod " li Pet . 5 , 5. ) og „ skjuler " sit Lys for de „ <b>Vise</b> og Forstandige " Math . 11 , 25. ) . Men naar Gud staaer et Menneske imod , da er det forbi med denne , da falder han af den ene Daarstab i den anden . „ Da faaer han en saadan <b>Visdom</b> " , siger Luther , „ at han anseer for Retfærdighed det , som er Synd , og den yderste Daarstab for den hoicste <b>Visdom</b> ; thi derhen Pleier Djævelen at stille Sagen , at jo fjernere Mennesket tommer fra Ordet , desto loerdere og / visere mener han sig at ocrre " . Men desuden pleie ogsaa ofte de sorteste Fristelser at aabenbarc sig blot som en Nyfigenhed eller Forvidenhed . Der er dels et Begjcer hos Mennesket efter <b>Kundskab</b> om det Forbudne , dels en Slangens Rost i Hjertet , som siger : det kan vcrre Dig nyttigt at blive viis ved Erfaring eller Ydmyg ved Synd ofll . Men ' ved denne Blanding af Sandhed og ' Logn har den gamle Slange fanget og myrdet mangen SM . ' En ' Opmuntring ' til Synd er altid af Djwvelen , ogsaa om han anvender en Tandhed dertil . Men vi gaa atter til Texten . Og <b>Kvinden</b> saa , at Trwet var godt , og det var hel lysteligt at see til og et onsteligt Trw til at faa Forstand af . Saa tog hun af dets Frugt og aad og gav sin Mand ogsaa med , og han aad . Her er det nu sammenfattet alt , som ' bevægede <b>Kvinden</b> til Synd : forst at „ Trcect var godt at cedc af og lysteligt at see til " . Der er nu selve Begærligheden eller Syndens Behag . Det Andet var , at Tnret var nyttigt , efterdi det gav Forstand . Der er nu Frugten af Djcevelens Prcrdiken . Det var nu saa langt fra , at hun troede Guds Trusel eller frugtede for nogen Straf , at hun tvcrtimod holdt det for nyttigt at

<h4>234</h4>Da Tidens Fylde tom , sendte Gud fin Son , fsdt af en Kvmde , fodt under Loven , at han stulde fritjsbe dem , som vare under Loven , at vi stulde faa den sonlige Udlaarelse ( Gal . 4 ) . Idet han indtræder i Verden , siger han til Faderen: Slagtoffer og Madoffer vilde Du ilte , men et Legeme beredte Du mig . See jeg tommer , der er skrevet om mig t Bogens Rulle , for at gjore Gud , Din Villie ( Hbr . 10 ) . Selve Offrene vare ikke det , Du vilde have , o Fader ! men min Manddom , mit Legeme , hengivet til Syndernes Forladelse <b>Kvindens</b> Sced , et sandt Menneske , hengivet for alle Mennesker til et Offer . „ Et Legeme beredte Du mig " ( ifolge Skrifterne) I Bogen ftaaer strevet om mig . " O , hvilket Rigdoms Dyb , bande i Guds <b>Visdom</b> og <b>Kundskab</b> ! Hvor übegribelige erc bans Domme , og hans Veie usporlige ! Den , som ikte Vidste af Synd haver han gjort til Synd foros ( Tcenk ! Tcenk ! ) paa det at vi stulle vlrre retfwrdige for Gud i ham ( 2 Kor . 5 ) . Det , som var Loven umuligt ( Mcerk ! det Loven ikke kunde vinde hos os ) idet den var lrafteslss formedelst Kjsdet , det gjorde Gud da han fendte fin egen Son i syndigt Kjods Lignelse og straffede saaledes Synden paa Kjsdet ( Rom . 8 ) . Saa elstede Gud Verden . Men hvem kan opregne alle disse trostelige Ord ! O , at Gud blot vilde give os visdommens og Aabenbarelsens <b>Aand</b> i sin <b>Kundskab</b> , vilde oplyse vor Forstands Oine , at v : mua forståa , hvad Haab han har kaldet os til , og begribe med alle Hellige , hvad Breden og Lcengden og Dybet og Hviden er af Kristi Kjcerlighed , som overgaaer al Forstand ! Vi

<h4>347</h4>det gode Land Kanaans Besiddelse . Men alle disse Pagter have havt deres Tid og deres Ende . For nu at blive ved vort Emne , Guds evige Pagt med sin Son , Guds evige Raad om vor Salighed , som sluttedes i Himlen for Verdens Grundvold blev lagt , og som i alle dets forundringsvcerdige Omstoendiglieder aldrig i Tiden kan ret begribes , aldrig nok betragtes stulle vi nu under Bon om <b>Visdommens</b> og Aabenbarelsens <b>Aand</b> beskue nogle Steder derom i den hellige Skrift , om maaskee nogen saliggjorende Lysstraale kunde derved blinke ind i vore Sjcele . Sporger Nogen , hvorledes man kan vide , hvad der besluttedes i Guds himmelske Raad , hvem der da har vcrret na ' rvcerendc og siden sagt os det , saa svares rettelig : Alle tre Personer , som deltoge i Raadet , have ogsaa kundgjort det . Faderen besogte strax vaa Faldets Dag vore forste Foroeldre i deres Blusel og Bedrovelse og forkyndte dem den besluttede Frelse ved „ <b>Kvindens</b> Soed " Kristns ( 1 Mos . 3 , 15. ) . Han aabenbarede sig siden ofte for Patriarkerne og Profeterne og fornyede Loftet . Den Helligaands Aabenbarelfer til Profeterne vare stundom saa tydelige , at en Apostel i det nye Testamente ikke kunde tale tydeligere . Men i Sceroeleshed har den „ enbaarne Son , som er i Faderens Sljsd , kundgjort os det " ( Joh . 1 , 16. ) . Derfor kaldes han og „ det trofaste og sanddrue Vidne " ( Aab . 3 , 14. ) hvilke Ord Buchne / forklarer sadledes : „ Han er mit Vidne til Guds Pagt med os ; han var velbekjendt med Sagen ; thi han havde overvceret Fredsraadet , hvori Beslutningen blev , at Sonnen skulde blive Menneske og igjenlose MennestesleeHten . " Og i Es . 1 , 9 siger Apostelen om „ vor Herres Jesu Kristi Fader," at „ han kundgjorde os sin Villies Hemmelighed efter den velbehagelige Beslutning , hvilken han forud fattede hos sig selv . " Men af alle Guos herlige Vidnesbyrd om fin Fredspagt ville vi blot anfore folgende : I Es . 53 Kap . bliver Profeten fort af Aanden til Apostlenes Tidsalder , taler i deres Navn og beskriver, forst Kristi Person , Skikkelse og Udseende , siden hvad han hos Faderen havde paataget sig at udfore for os , og tilsidst hvad Faderen lovede ham til Lon for hans Arbeide . Efterat Sonnens Udseende er blevet tegnet i v. 2 og 3 , saa folger : 4. „ Visselig haver han taget vore Sygdomme vaa sig og baaret vore Piner ; men vi agtede ham for den , som ( for sin egen Skyld ) var plaget , slagen af Gnd og gjort elendig . " 5. „ Men han er saaret for vore Overtrædelser , er knuset for vore Misgjerninger ; Straffen laa vaa ham , at vi skulde nyde Fred , og vi have faaet Lcegedom ved hans Saar . " 6. Vi , vi fore alle vild som Faarene , vi vendte os hver til sin Vei ; men Herren lod alle vore Misgjerninger mode ham . " Dette om , hvad Sonnen havde paatagct sig ; men i u . 10 — 12 staaer , hvad Faderen vaa sin Side lovede Sonnen . Der staaer nemlin : 10. „ Naar han haver givet sit Liv til et Skyldoffer , stal

<h4>735</h4>saa stal han svare dertil for sin Gud ; men det berettiger : kke Dig til at betale hans Uret med en ny eller at foragte ham derfor . Dn bor tillige tage Dig vel i Agt , at Du ikke med Ovforfel , Ord eller Gjerninger forulemper eller fortorner ham ved en uvillig og trodsig Aofcerd . Naar Gud vil , er han mcegtig til at soette ham fra fin Plads ; men saalcenge Gud fordrager ham , bor ogsaa Du gjore det ; thi hvormeget han end gjor Dig imod , saa gjor han dog Gud titusinde Gange mere imoo . Visselig er det ingen let Sag ei blot at fordrage , men tillige agte faadanne Forceldre eller dem , som ere i Forældres Sted , naar de gjore Alt for at undergrave denne Agtelse hos deres Born og Undergivne : derfor er Aandens Kraft og en guddommelig Taalmodighed fornoden , fom Syrcck siger : „ LEr Din Fader og <b>Moder</b> med Gjerning , med Ord og med Taalmodighed . " Taalmodighed er en Dyd , fom er nodvendig ved Omgang med Mennesker i Almindelighed , efterdi man overalt scettes paa Prove formedelst Andres Feil og Skrobeligheder , som maa fordrages og boeres med Taalmodighed , eller ogsaa Kjcrrligheden bliver kold ; men i Scerdeleshed er Taalmodighed og Fordragelighed hoist nodvendig , naar det gjcelder Forceldrcne ; thi foruden det , at det fjerde Bnd tilegner Forældrene faadanne Fordringer paa os , som intet andet Menneske har , saa har heller intet Menneske bevift saadaN Taalmodighed mod os som Forceldrcne , hvorfor heller ingen har fortjent vor Taalmodighed saaledes som de. Taalmooighed er fornoden ; thi medens Borncnc voxc til og med hver Dag voxe altmere ind i Verdens ncervcerende Skik og Maneer , saa voxe Forceldrene fra deres Tid , eller Tiden lober med fine nye Skikke og Vaner forbi og bort fra dem , hvorved Born og Forceldre i tusinde Smaating tildels blive fremmede for hverandre. Naar nu Forceldrene tillige blive flappe og ved Alderdom tabe i Sjcelstrcefter , Forstand og Hukommelse , ja „ blive Born paany " samt under alt dette <b>vise</b> Vrantenhed og dcmrligt Lune , for hvilket ingenting , fom Narnet gjor , er ret tilpas , og de dog uindstrcenket ville udouc deres Forældremyndighed over Barnet , da gjcelder det at fornegte sin egen <b>Visdom</b> , ja i Alt at fornegte sig selv , om man nu stal kunne „ cerc Fader og <b>Moder</b> med Taalmodighed . " Er nu ikke Guds Acmd boende i Barnets Hjerte , saa at det drives af denne hellige Frygts og Kjcerligheds <b>Aand</b> , da ganer det , fom man saa ofte seer , Barnet trcettes ved Forceldrenes vrantne , besvcerlige Adfcerd og begynder ikte blot at foragte dem i Hjertet , men ogsaa at lade denne Foragt aabenbare sig i dets Ovforfel , Forceldrene til dybt faarende Fortornelse. Ja endog der , hvor Guds Acmd boer i Hjertet , men man enten ikke agter dens hnlde Paamindelser , eller maa ved tidligere feilagtig ' Opdragelse er olevcn ilde vant , kan der blive fyndet ' paa den forskrækkeligste Maadc i denne Henseende. Dette er iscer Tilfceldet blandt Almuen , som i saa Henseende

<h4>763</h4>Alvor . Men her komme vi atter til et Emne , til hvis Fremstilling vi finde ingen Ord kraftige nok . Hvor hvor ligger den ncermeste Skyld til alt det Onde , som der med Grund klages over hos Born , Tjenere , Undersaatter , ogTilhorcre ? Hvor , om ikte fornemmelig hos dem , som erc satte til deres Vogtere og Opdragere? Hvad Bornene angaa , saa have de nisten alle deres Vanarter af . Forældrene , ikte blot i Arv ved Fodselen , men ogsaa siden ved Forældrenes Forsommelse af de Midler , Gud har givet dem til Hjcclp imod denne Fordærvelse . Det er forskrækkeligt, hvilten Synd det Barn bcgaacr , fom forulemper eller bedrover fin Fader og <b>Moder</b> ; men hvor utallige ere ikke de Tilfalde, i hvilke de arme forulempede Forceldre blot kunne slaa sig for deres Bryst og betjende : „ Vi lide det , vore Gjerninger have forskyldt ; vi have ikke opfodt vore Born bedre ; vi hoste nu den bittre Frugt af , at vi ikke tidlig indplantede god Seed i den engang bolelige Jordbund at vi ikke efter Herrens Bud have opfodt vore Born i Tugt og Herrens Formaning , " at vi lttc i broendende Bonncr Dag og Nat have baaret dem frem til den ene almægtige Hjoelper . " Ligeledes er der meget at klage over Tjenestefolks Vanart , Forsommelighed , Uhoflighed , Uenighed , Mangel paa Samvittighedsfuldhed o . m . dcsl . men om Husbondfolkene daglig forte dem frem til Familiealteret , daglig helligede Husets Aaud med Guds Ord og Bon og selv i al deres Omgængelse beviste Gudfrygtighedens Kraft , saa skulde de sikkert faa mindre at klage over hos Tjenerne . Forsom Husandagten , men lid ogsaa siden taaligt , at Djcevelen regerer i Huset ! Endnu har Ingen opfundet et kraftigere Middel til at bibringe Folket en god <b>Aand</b> end Guds Ord og Bou . Som Luther skriver til dem , der vare vante ved Papisternes Midler , Rogelser , Vievand , Korstegn m . m . : - „ Du stal aldrig finde noget stoertere Rogvcerk mod Djeruelen , end om Du sysselsætter Dig med Guds Ord , taler og synger det og beder derover . Det er visselig det rette Vievand og Tegn , for hvilket han flyer og hvormed han lader Ng fordrive . " Saadant burde Husfcedre og <b>Husmodre</b> tcenke — Fyr Sjcelesorgere er det ogsaa bittert at see saamegen Vanart hos Tilhorerne : det hellige Embede bliver svarlig foraatet l denne Tid , og Folket er ugudeligt m . m . Men skulde ikke " de stefte af disse Lwrere slaa Oinene ned og for Gud og Mennesker betjende : „ Vi , vi have syndet og voeret ugudelige : vi have ikke holdt vort Embede i VEre med Loerc og Liv ; vi ' have ikke efter Apostelens Bud holdt ved itide og utide at overbevise , straffe og formane med al Langmodighed og <b>Lcerdom</b> ikke villet vcere Hyrder , men Herrer over Hjorden " osv. See saaledes skulde Enhver gaa til sig selv og begynde det Heles Forbedring med stn egen . Men vi maa indskroenke os til blot at tale med Forcrldre og Borneopdragere . Er Du Fader , <b>Moder</b> , Fosterfader , Borneloerer o , betoenk , hvilket dyrebart og vigtigt Kald , der er Dig 19 «

<h4>970</h4>Grad , som Aanocn er villig , hellig og Vaagen , i samme <b>Maal</b> fole vi ogsaa Fordaervelsen stcerk og paatroengeilde . Syndens Erkjendelse beroer blot paa , hvormegct Gud betyder sor Mennesket; har han en virkelig hellig og sandfærdig Gud for sig , da kan han fortceres , som alle Helliges Exemftcl viser . Eller findes der en eneste Kristen saa hellig og alvorlig , at han endog kun cn eneste Time opfylder , blot hvad det storste og ypperste Bud fordrer ? Vi sporge dem , som ere mest alvorlige og have mest af Guds <b>Aand</b> og Kraft . Men hvad er al Fromhed , om Du ikke holder det storste og ypperste Bud ? Det Bud fordrer , at Du skal elske Herren Din Gno af Tit ganske Hjerte af Din ganske SM , af Din ganske Styrke , af Dit ganske Sind , og derhos , at Du stal saa helt og aldeles forlade Dig paa ham alene og frygte ham , som hans guddommelige Magt og Trofasthed fortjener. Er det ikke sandt , at om Dn saaledes elsker Gud af Dit ganske Hjerte og forlader Dig paa ham alene , saa at Gud alene er det hoieste <b>Formaal</b> for Din Kjwrlighed , Din Tillid og Frygt , saa maa Din Sjcel vcrre i en evig Ro , i den uforanderlige Nydelse af dette eneste Gode , Dn begja ' rer , den uforgængelig / Gud ? Da maa Du tro , at ikke et Haar falder af Dit Hoved uden Din himmelske Faders Villie , at ikke det mindste kan hamdes Dig , ikke et Ord , ikke et Blik saare Dig uden Din himmeNe Faders Villie . Naar Dn da intet andet elsker end ham og hans Velbehag, saa maa jo deraf folge , at Du alletider er lige rolig og lykkelig , hvad der end vederfares Dig , blot fordi Du vced , at Alt kommer fra Din Gud , og Du elsker ingeuting andet end ham og hans Velbehag . Hvor er nn det hellige Menneske , som holder dette Bud ? Vi ville nu tale med en saadan . Er Du lige rolig og lykkelig , om Nogen tager al Din Eiendom fra Dig og Du nedsyuker i Fattigdom og Mangel ? lige rolig og lykkelig , om Nogen derover D ' ig Dit gode Navn og Rygte , om Du bliver cerelos , foragtet og afskyet af alle Mennesker for hele Dit efterfolgende Liv ? lige rolig og lykkelig , om en svcer Sygdom , ja en Morder forkorter Dit Liv ? Om det er sandt , at Du elsker Gud af Dit ganske Hjerte , af Dit ganske Sind og af alle Dine Kroefter og derhos troer paa ham alene , at ingeuting <b>moder</b> Dig uden hans Villie , saa maa Du nodvendia . vcere lige rolig og lykkelig ved alle disse Men maastee det er saalangt derfra , at Dn tvertimod bliver fornroliget over et ganske ringe Tab ! at om Dn blot faaer at vide , hvorledes man har talet ilde om Dig eller blottet nogen af Dine Svagheder , det forstyrrer Din Ro for hele Timer og Dage ? Ja maastee blot et foragteligt Blik foruroliger Dig ! Hvorledes elsker Dn da Gud alene og hans Velbehag ? Ja foler Du , at Du rigtig varmt elsker Din Gud , saa at Dine Tanker ere bestandig hos ham ? maastee Du tvertimod meget livligere elsker og tamter paa noget Menneske ! Men videre : Er det ikke sandt , at vi , som ere gjensoste med Guds Sons Blod fra alle vore Synder , fra Dsden og fra Djcevelens

<h4>978</h4>Synden i selve Hjertet , jeg elsker Synden ! " thi da fornemmes Lyster og Begjcermger nemlig som en Kjcerlighed til Synden . Efter den nye , hellige <b>Aand</b> hader og fordommer jeg disse Lyster , denne Kjodets Kjcerlighed til Synden , men maa dog fole den hos mig . Men tamt Dig ' nu en virkelig Hellig , en , som „ horer Kristo til " ; kan Du tamke om en saadan , at han har onde Lyster og Begjcermger ? Nei , det er urimeligt , synes Du ; men her seer Du nu , at Apostelen siger det . Det er dette Forhold , som forklarer , at samme Mand , som har det stjonne Vidnesbyrd , at han var „ en Mand efter Guds Hjerte " ( 1 Sam . 13 , 14. ) kunde falde saa forskrækkeligt for onde Lyster og Begjcermger , at om ikte Guds underlige Naade havde ovsogt og reddet ham , saa havde han vceret evindelig fortabt ( 2 Sam . 12 ) . En anden Mand havde Guds eget Vidnesbyrd , at „ hans Lige ikke fandtes i Landet , han var retfcrrdig og from , gudfrygtig og flyede det Onde " ( Job . 1 , 8. ) men da Fristelsen blev altfor stcert , faldt han i saa stor Utaalmodighed , at han forbandede den Dag , da Gud lod ham og den Kjcerlighed , hvormed han opammcdes ( Job . 3 ) . Den , som nn Gud selv kalder „ from , retfcerdig og gudfrygtig " , maa sikkert v crre det i Sandhed, men ikke er han derfor fri for Kjodets Svaghed ! Abraham, som kaldes alle Troendes Fader , og med hvem Gud talede som en Ven med en Ven , han var visselig en storre Helgen end nogen af os ; men faa fuldkomment kunde han ikke elske Gud alene og forlade sig paa ham , uden han jo bar Frygt for sit Liv og sogte selv at redde det , da han sagde om sin <b>Hustru</b> , at hun var hans Solter ( 1 Mos . 12 og 20 Kap . ) . Hvad skulle vi nu lcere af saadanne Exemvler ? Aldeles ikke , at Enhver , som har Synden tilfcelles med de Hellige , er en Kristen. Kun den er en Kristen , som ogsaa i Omvendelse og Tro er lig de Hellige , og til Grund for vor Tro og Benaadning fordres noget helt andet end de Helliges Skrobeligheder ; til Trost mod Syndens Skyld og Dom forslaaer intet mindre end Kristi Fortjeneste . Endnu meget mindre stulle vi tage den Mening heraf , at Synden ikke er farlig , efterdi den ogsaa sindes hos de Hellige . Nei , en saadan Trost ville vist de falske Aander tage , som misbruge alt Guds Ord til deres egen Fortabelse ( 2 Pet . 3 , 16. ) og dog maa alt Guds Ord bruges ; men de Troende have en <b>Aand</b> , som ogsaa med de Helliges Syndefald advarer dem desto mere for Uagtsomhed , Sikkerhed og Overmod , hvorfra ethvert Fald hidledes . Men den <b>Lcerdom</b> har Herrens Nand villet give os , naar han har ladet de Helliges Tynder optegne og sagt , at „ den ganske Skrift er indblcest af Gud og nyttig " den <b>Lcerdom</b> , at vi aldrig stulle tro , at noget eneste Menneske , som endnu gaaer paa Jorden , er fri fra Slangenden , fra Synd og Feil . Deraf skulle vi lcrre Idmyghed og Gudsfrygt og at have al vor Trost i Guds blotte Naade , men ogsaa lcere ikke at lade os berove vor Trost i Kristo for vore vedh ' crngende Strobebelighedcr,

<h4>1106</h4>hans Rige ? Apostelen siger so . 12 ) ; at I paa den Tid vare uden Kristo , udelukkede fra Israels Borgerret , og fremmede fra Forjættelsens Pagter , udenHaab og uden Gud i Verden , hvilket ikke blot indtrceffer paa Hedningene , men ogsaa her i Kristenheden , og er saameget forskrcekkellgere her , som vi leve midt i Rigdommen af Guds Ords <b>Kundskab</b> , ere engang optagne i Israels Borgerskab og ere faaledcs bundne ved al Ansvarlighed . Dette er nn Verdens Tilstand , og saaledes , siger Apostelen , ere Vi nn alle af Naturen Vredens Born ( v. 3 ) . Naar vi nu have scet , hvorledes hele Menneskeheden er af Naturen <b>aandelig</b> dod og som et Lig intet fornemmer , men fores af Morkeis Fyrste efter hans Villie og Kjooets Begærligheder , alt under den mest uforstyrrede Doosro , saa maa vi vel sporge , som Herren spurgte Profeten , da denne faa ud over den ftore Dal , som laa fuld af de Dodes Ven : „ Du Menneskebarn ! troer Du , at disse Ben kunne atter vorde levende ? " Og vi fole os tilboielige til at give Profetens Svar : „ Du veed det , Herre , Herre ! " ' - Tcenk da , hvilken mcerkvcrrdig Tildragelse paa Jorden, naar et aandeligt Liv opstaacr , naar „ en <b>Aand</b> bleeser fra Herrens Ansigt og giver Liv / saa at det begynder at rore sig iblandt de Dode ! hvilket Scerfyn , naar et Menneske , som i tredioe , fyrretyve , halvtredsenstyve Aar har levet aldeles dod for alt <b>Aandeligt</b> , men idel forviste , forfængelige Tanker og Be- , strcrbelser , paa engang vaagner til Omsorg for Sjælens Frelse ! Tamt , i et Hus , hvor Fader , <b>Moder</b> , Born og Tjener have levet tilsammen i saa mange Aar ial Ro blot med jordiske Anliggender, blot fandselige Sorger og Glceder naar der aldrig en eneste Gang i dette Hus har ucerct Sporgsmaal om , hvorledes man har det med Himlens Gud man har blot efter gammel Stil loert sin Kristendom og iagttaget Kirkens Brug , paa samme Maade som Kirkeklokkerne hver Sondag blcve fatte i Bevægelse; men Hjertets Omvendelse , Gjenfodelfe , sand Forening med Gnd saadant kom aldrig under Sporgsmaal — og Aarene gik fremad med deres Omskiftelfer , Vaarcn med sine nye Bestjceftigelser , Sommeren med fine , Hosttidcn med sine men Sjcelen Sjcelen blev aldrig <b>Formaal</b> for nogen Omsorg ; og nu begynder der i det Hus at opftaa et Sporgsmaal , imod hvilket hele Verden bliver et Intet : Hvad stal jeg gjore , at jeg kan blive salig ? Nu sees den gamle Fader eller <b>Moder</b> for forfte Gang i sit Li o urolig for sin Sieels Frelse fra den evige Fordærvelse ; nu faaer man saadanne Ord at hore , som aldrig for ere Horte i dette Hus , og snart hores nu Lammets Wre at synges af Munde , som hidtil have vcrret <b>aandelig</b> stumme , " som have talet tusinde og Millioner Ord om allehacmde Smaating , men aldrig en eneste Gang i hele deres Liv have talet et godt Ord om deres Frelser . Hoenk , ien By eller Menighed , hvor Ingen i Mands Minde har havt nogle aandclige Bekymringer , men alting har gaaet sin jevne , naturlige Gang under blot jordiske

<h4>1136</h4>forst cr udtroettet af min egen Villies Ondstab og siden saa gjennemtrcrngt , dels af Guds Storhed , at hans Villie er mig mere end hele min Salighed , dels af hans Kjcerlighed og Liflighed, saa at , naar blot hans Villie steer , er Alting vel , da har jeg et faadant Hjerte , fom virkelig elsker Gnds Villie . Her maa nn Enhocr give Agt paa fin Tilstand . Vi have her atter et Stytte , som « åbenbarer vort Hjertes Inderste . Vi erindre atter om , at Bonnen maa vcrre et Udtryk as Menneskets egen Omsorg , Begjwring og Anliggende . Ikte hvemsomhelst stal da kunde bede denne Bon . Vi tale dog ikte om den store Hob , som lever aldeles frit „ efter Kjodets Begjcrringer ; " men der er endda to Slags Mennesker , som ikke ret kunne bede denne Bon . Forst er der Nogle , som visselig bede den ret alvorligt og flittigt , efterdi hele deres Salighedshaab i sin inderste Grund beroer paa deres Opfyldelse af Guds Villie disse „ Tjenestekvindens Sonner " , hvoraf nogle med tung og moiefuld Kamp sutte efter Naade til at kunne gjore Guds Villie , medens derimod andre iblandt dem , just ved Tilfredsheden med deres Gjoren og Beden , have en <b>vis</b> Lyft dertil , ja en <b>vis</b> Tro og Kjoerlighed , men altsammen grundet just paa denne deres egen Fromhed . Da de dog aldrig ere „ ved Loven dodc fra Loven , " aldrig ere blevne helt tilskamme , fortabte og fordomte i sig selv eller ere blevne salige ved Kristi Gjerning , Bon og Lidelse aleue , saa hore de endnu til „ Tjenestekvindens Sonner , " som ikte for Guds Skyld , af Kristi Kjcrrlighed , men for deres egen Skyld , deres egen Fromhed og Salighed jage efter at gjore Guds Villie . Og „Tjenestekvindens Son stal ikke arve med den fri <b>Kvindes</b> " ( Gal . 4 , 22 — 31. ) . Men der er ogsaa en Slags evangeliske Folk , som ikke heller kan bede denne Bon disse Evangeli lose Tilhængere, som gjerne hore det , der lyder lifligt og vel , samt have en meget uforstyrret og jevn Tro . men en saadan Tro , som ikte medforer noget helligt Sind , saa at det angriber deres gamle Menneske og gjor , at Guds Villie bliver deres Levnetslov , bliver <b>Formaal</b> for deres Hjertevenner , men de leve aldeles fri for denne Omsorg for Guds Villie ; de have i deres Liv nogen Form af Kristendommen til et vist bestemt <b>Maal</b> , med hvilket de noies , og derfor ere de siden fri fra denne Omsorg . De lcese Bonnen : Slee Din Villie ! med eller uden Tanke , men om end Tanken folger med Ordene , er det dog ikke mere , end at det bliver en blot Tanteosn . ikke Hjertets eget Anliggende . Du . som lceser dette , stands dog en Stund for Guds Ansigt , for de Oine , som skue Hjerte . Tanke og Forsoet . Hvorledes er det med Dig i dette Stykke ? Du kan vel vide mZd Dig selv , om Du pleier at have saadan Omsorg , omgaaes med saadanne Sukke og Bsnner : „ Gud , viis mig Din Villie , hjcelp mig at gjore Din Villie ! " Det er jo umuligt , at den Helligaand kan bo i et Hjerte uden at opvække saadan Omsorg derinde . En Kristen kan vel ofte overfare Bonnen : Slee Din Villie ! saa adspredt , at han ikke engang

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112301011'target=_>Hansen, Maurits, 1856, Mauritz Hansens Noveller og Fortællinger</a><h4>287</h4>En forlegen Stivhed udbredte sig over det lystige Selstad ; thi uagtet dette bestod af Bygdens anseeligste Vender , indged dog Sergeant Wilhelmsens dannede Vcesen og bydende Udvortes en <b>vis</b> Wrefrygt , der ikke passede til den jevne Ligefremhed , som udmcerker vore Bsnders lystige Gilder . Men denne Vilhelmsen besad lige saa megen Lethed i sin Omgang som sin <b>Dannelse</b> , og snart vidfte han at erstatte den tabte fortrolige Stsjen ved at give Selstabet et hsjere Sving . De unge <b>Piger</b> betragtede den velvorne , smukke Karl med Velbehag , og da han havde danset den forste Fejer med Bruden , stog deres Hjerte hejt under Livstykket af crngstlig Forventning om , hvem der herncrst skulde opbydes af ham til den Vals , han bestilte hos Musikken .

<h4>1054</h4>Vi have nu lest vor Gjeld til Lceseren , Ragnhilds Historie angaaende, og stulle vende tilbage , eller egentlig talt atter de tre Aar fremad , til Leiners Ophold paa Thorsrud . — Margrethes Undervisning beskjeftigede alvorligen baade hende og hendes Fcrtter . Langt fra at besidde Ragnhilds ualmindelige Geni og flammende Begjærlighet » ester et bestandig fjernt <b>Maal</b> , lcerte Margrethe aggregationsvis , uden Vegejstring , men med klar <b>Bevidsthed</b> , hvad hun troede at burde vide , for at vcere tilfreds med sig selv . Margrethe blev den , hun var , — stille , ordentlig, virksom . Det var hende fuldkommen nok , at hun var sig selv fine flere Kundsiaber bevidst ; i det hejeste glcrdede hun fig ved , at Lemer var tilfreds med hendes Fremsiridt . Hendes Sjcrl var alt for moden til at betragte disse Tilgifter til <b>kvindelig</b> Fuldkommenhed som en Forgyldning, hvis Glands siulde siaffe hende et hojere Vcerd i Mcrngdens Omdpmme. Vi kalde dem efter hendes egen Mening Tilgifter , disse Kundfiaber ; thi saa hovedsagelige som de ere , hvor de fra Barndommen af indflettede med den hele Tilværelse frembringe den alsidigen dannede <b>Kvinde</b> , — saa lidt udgjere de noget Vcesentligt , naar de blot tilhcenges , for af og til at bruges .

<h4>1448</h4> Hvor i Satan har I den Oberst fra ? Er han enda Lsjtnant , saa er det meget vel , for mere kalder han sig ikke selv . Jo han har nok Lyst til at faa Fingre paa hende ; det tror jeg übandet . Men hvad Forftrgelsen angaar , det sial vcrre min Sag ; og faar jeg fat paa Personen , saa sial jeg nok <b>vise</b> den hoje Herre Vintervejen . Til Vaaren , min kjerre Ragnhild , henter jeg Dig af . Du stal komme til brave Folk i Sverige , hvor D « kan faa lidt <b>Lcerdom</b> * * ) i Dig . Ved den Tid sial jeg siaffe jer en skikkelig Tjenestetss ; for I ved nok , jeg er ikke faa lige fornojet med , hvem her kommer . Desuden sial Mangt og Meget herefter blive sat paa en ganske anden Fod end hidindtil . " — Birthe taug for det ferste stille , og Ragnhild gik beklemt ud i den frie Luft . Hvad der alene gav den arme <b>Pige</b> Mod , var den lange Opscettelse . Naar nu Faderen var borte , mente hun . og Ridderen saa vilde holde sit Loste , saa stod jo den hele vide Verden aaben for dem med alle sine Herligheder .

<h4>2127</h4>Det var en smuk Aften , allerede halvmerkt . Egnen havde en <b>vis</b> Lighed med et af Elauren beskrevet Schweizerlandstab , og jeg fandt mig derfor stemt til at hendromme nogle paa en Brink , imedens jeg lod Skydsen rulle afsted til Gjestgivergaarden . En r > > Ue clauce , som jeg med megen Meje har lcert at spille paa , blev fremtagen . Men dels duede Ekkoet ikke , dels forstemtes jeg ved Forestillingen cm , at jeg naste Aften stulde se min norsie <b>Pige</b> , der ikke med al sin Sjcrls- og Legems-Unde har Lighed med de clanrensie <b>Piger</b> , cg omfavne min Sester, der er vittig , hvor Mimili er selsom , og har Taarer for Ting , hvoraf Mimili ler . Pludselig vcrkkes jeg af mine mismodige Tanker ved en Raslen i Bustene . Jeg ser op. Fra en Sti trceder ud paa Landevejen en rank crtherist Kvindestabning . Hun drejede saa pludselig forbi mig , at jeg ikke bliver Ansigtet var , men kun bagfra har Synet af den nydeligste Pigesigur , der med de letteste Trin svaver op ad Bakken. Et Blik paa den fugtige Jord viste mig den sijonnefte <b>Dannelse</b> af en Sko , et andet paa den forsuindende <b>Pige</b> , den smagfuldeste Bondepigedragt. Som et Lyn var jeg asfted ; aandeles stod jeg ved Siden af Pigen , der forundret standsede og drejede sig halv om , idet et halvforstrcrkket,,God Kveld ! " i Eelvtoner foer ud fra det bankende Bryst . Hun bsjede Hovedet ned , og en bred Straahat ssyggede for hendes Ansigt . Jeg havde aldrig vcrret i en Situation , der saa meget lignede Claurens Heltcs ; derover blev jeg Ilge saa forlegen som overlastet . Med usikker Stemme ledte jeg ester et :  Det er et smukt Vejr i Aften , vakre Gjente ! "

<h4>3403</h4>jeg foler mig saa lidet stikket , uagtet man behager at forsikre mig om det modsatte . Om min Elev , min lille , fortryllende Emilie , tcrnkte jeg , da vi talte om skriftlige Meddelelser , at jeg iscer skulde have meget at fortcrlle Dem . Men , hvorledes sige det ? Uagtet hendes hele tryllende Ynde , der bedouer Mcendene og tiltrekker <b>Kvinderne</b> , har jeg intet Vist at sige . Dyden er ikke Sum af en <b>vis</b> Mcrngde Dyder eller Karakteren af hundrede karakteristiske Trcek . Det er en underlig Modsigelse , og skulde den overftres i Filofofernes Sprog , kom det til at hede absurd nok : det Hele er ikke ligt sine Dele . Min Plan for Emilies <b>Dannelse</b> er udkastet; men , som jeg engang i mine Semandsdage vilde udtrykt mig . Fortoningen er ikke Land . Gud give mig Kraft til at udvirke noget ret Godt af det meget Gode ! Hr. Desmouil den yngre ! Min kjcerestk Adelaide , hvad skal jeg sige Dem ? De har Ret i at sige , at han er meget elskverdig . Han er i mine Bjne en fuldkommen Mand . Ikke havde jeg troet , at mit Hjerte endnu engang skulde kunne overaskes af en Folelse, som er saa farlig for vor Rolighed . Men da jeg nu engang har sat mig for at vcere fuldkommen oprigtig imod min Adelaide , saa modtag her Offeret af min Oprigtighed i Tilstaaelsen af min Svaghed . Det er denne Svaghed , som gjor den unge <b>Pige</b> guddommelig og den gamle — nu vel , jeg er en saadan — latterlig eller dog ynkverdig . Det er udsagt , De ved det , — jeg elsker ham . "

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081903028'target=_>Hippel, Theodor Gottlieb von, 1861, Om Ægteskabet</a><h4>30</h4>Gaffel paa hoire Side og Brodct paa venstre ; ikke at sige Du til din Fader , men n a a d i g e Papa , v cc r s a a n a a d i g at hore ; ikke at sige til din <b>Moder</b> , kjcere <b>Moder</b> , Du er den bed ste Q vinde i Verden ; men naadige Mama , De er pyntet som en Engel ; naar dn <b>moder</b> dine Icevnlige , at stjcele dem det Sporgsmaal nd af Munden , hvorledes be fin der De Dem ? og naar det er Fornemmere, at udbede dig Tilladelse til at sporge til sammes Velbefindende eller hoieste Velgaaende ; at aflevere et Visitkort , skjsndt Folkene sidde i Vinduet , og man dog ved Hjcelv af en romersk Fiction maa troe , at Herren og Frnen ere reiste paa Landet , og flere saadanne Ammesager . — Diogenes var ester min Mening bleven rigere , da han loerte at indsee , at man ogsaa kunde ose Vand med Haanden . Naar Staten holder sine Borgere under saa slette Forhold , at de ikke kunne opdrage deres Born , er det da ikke dens Pligt i egen hoie Person at sorge for hvilket Tab for Staten , naar den mister Tusind Mennesker , som kunde due til Borgere , for at bevare Hundrede ertrafiint opdragne Borgere , men som i tredie og fjerde Led vist ikke vare istand til at frembringe Noget i deres eget Billede ! Den , som ikke indseer , hvad jeg vil sige , med ham vil jeg ikke trcettes ; og den , som tåger mig det ilde op , at jeg her , og ikke ellers , har gjort lange Perioder , han skal ikke lcese mig . Intet er mere unaturligt , end at forberede sig saa lcenge til en Ting , som er fuldendt i saa kort Tid ; thi skjondt jeg giver den skjelmske Digter Net , at Jupiter som Gud ikke vilde have besluttet at <b>vise</b> Danae en menneskelig A3rc , hvis hun ikke havde vceret indespcerret ( thi DuleinLer , som smcegte under Trykket af Fortryllelser , forstccrke Lcengsel og Kjcrrlighed , ligesom Vinden Ilden ) ; saa maa dog Alt have sit <b>Maal</b> og Meed , og Eet er Magelighed , som frembringer Vcemmelsc , et Andet er et Bjerg af Vanskeligheder, der falder ynglingen z . , ^ bedste Lunger besvcrrligt. En Tyv , som er vant til at bryde Dore op , gaacr rigtignok forbi et aabent Huns , men derfor er han ogsaa en Tyv ; og de fleste Mennesker , skjondt de ikke ere Tyve , ere dog ligegyldige mod det , som kommer dem imode , men lsbe ester det , som flyer for dem ; kun maa man ikke lade dem blive aandelose ; thi det var det samme som at

<h4>111</h4>denne Pligt , fortjener ikke , at hans Enke stal have ham i stadigt Minde , skjondt han ellers har Ret dertil . Dottrene bortspille saa let og saa gjerne Faderens Tilnavn, ofte for en ussel Ret Lindser , og ville kun lidt eller slet ikke tåge i Betenkning fra Barnsbeen af at give Slip derpaa . Hvis min Nabo hed Heidefeld , saa kunde hans Datter kaldes Froken Mine af det Heidefeldske Huus . Men havde hun i det halvtredsindstyvende Aar , efterat vere anbragt i en Familiestiftelse , Lyst til at antage Navn af Amadeus Kreuzberger , saa lad hende have den Fornoielse , ligesom Froken 6 6 Gournay , som , faa smuk hun var , ansaae det for en 2 Ere , at fore Montaignes Navn . Et <b>Fruentimmer</b> er en Consonant , som man ikke kan udtale uden Maud ; hvor uretfcerdigt altsaa det er af Lovene at befale , at et <b>Fruentimmer</b> nodvendigviis skal tåge en Mand , som er hende jevnbyrdig , falder let i linene . En Mesalliance er i de fleste Tilfelde et utaaleligt Begreb . Romerne havde ooutulxzrinum , matrimonium , ooimudium , eller iiuMi » « oi ^ uFiuin eller luatrimonium iu Lpseie ; og alle disfe Veie forte til eet og samme <b>Maal</b> , nemlig Trolovelse og Bryllup . Tydfkerne , fom overhovedet holde mere af Ting end af Ord , forene alle disfe Navne i eet , nemlig VEgteskab , og indromme fammes Fortrin ogfaa til Mennesker, som vi ellers indskrenke i de borgerlige Rettigheder , idet Brude- , Barsel- og Dodssengen aftvinger til Mre for Mennesieheden Alle og Enhver den Bekjendelse , at vi ere hverken meer eller mindre end Mennesker . Hos Romerne indsaae man ogsaa snart , at altfor mange Undtagelser fvcrkke Regelen , og allerede Antoniuus Caracalla gav de Fremmede M8 comnudioruin og Meget blev bragt under een Hat : men ikke Alt vilde , formedelst de romerske Indretninger , lade sig bringe under denne Hat . 1,6 x et , ? kpia indfkrenkede Senatorerne og deres Born , der ikkemaatte indgaae A3gtefkab med Frigivne ; og gid dog Tydfkerne , som <b>vise</b> den romerske Ret saa megen ustyldig Lydighed , aldrig uden Nodvendighed vilde overdrive denne Foielighed , ikke at agte Love , fordi de ere Love , men fordi de ere retfcerdige , > og fom i mange Tilfcelde saa ogsaa ved LEgteskabet , holde sig til Naturens Kilde , som hverken foragter eller forsommer Nogen , der ikke nedverdiger sig selv .

<h4>244</h4>ved forskjellige Spiser og ved yppig e Fornoielsers Solsc ismer , som Lucian kalder overdrevne Loekkerier i Spise og Drik , fordi man scetter Priis paa dem ikke for deres Velsmag , men fordi de ere saa vanskelige at opdrive , og man opsoger dem fra de Karpathiske Bjerge ligetil Oceanets Kyst . Der findes blandt Hinduerne Classer ( B anianer) som kun nyde eet Slags Spise , og ester hvilke de engelske Matroser , naar de blive satte paa Grsnt og Plantespiser, kalde en saadan Faste et ß an ia n liv , og et saadant Maaltid et B anianmaa ltid ; og disse Mennester ere langtfra ikke saa lcekkersultne og graadige som saadanne , der ikke holde noget Maaltid uden i fem Acter og sammensmelte Tunger , Hjerne , Mcelk , Lever og andre fortrinlig yndede Dele af utallige Dyr til en Ragout , og fkjoerpe ovenikjobet disse Epigrammer ved tusind pirrende Saucer og Kryderier . Hvis Noah , der tillykke endnu ikke var kommen saavidt i sin Tunge-Erfaring , at han forstod sig paa Vinen og dens Krcefter , havde kjendt en faadan Ganekildren ; saa vilde han i ser Maaltider have spiist alle Arkens Dyr , og vi vilde have mistet dem , uden at vide hvorledes . Dog ikke alene Staten , men ogsaa dit eget Huusvcesen burde opmuntre Dig til Troskab i LEgtestanden . Du unddrager dine Born det , som Lovene have tilkjendt dem , og skjcender Dig selv , da Du istedetfor at avle Sonner og Dottre , lader Bastarder komme til Verden , som Du maa skjule for Allverden , som din <b>Kone</b> afskyer , og for hvilke Du i ethvert af dine cegte Born opdrager en dsdelig Fjende . En Mand , der foragter sin <b>Kone</b> , foragter ogsaa sine Born ; thi han forbander den Anledning , som har gjort ham til Fader , og vcenner sig til en Unaturlighed, som er den sletteste Vane af Verden . Elskoven har blot Nydelse til <b>Maal</b> og er en forbigaaende Trang , en Spadseregang til Motion ; i LEgteskabet er dette ligeoverfor andre vigtigere Forpligtelser en ringe Ting , og naar man gaaer , vil man til et bestemt Sted . Vil Du ausee din cegteviedeHustrue blot som en Gjenstand for Venskab , saa havde Du dertil ikke havt et <b>Fruentimmer</b> nodig ; eller holder Du et <b>Fruentimmer</b> for nodvendigt til at yde visse Tjenester , som iscer gaae ud paa Vcerelsernes Reenlighed og din Linnedvadsk, — hvorfor tog Du da ikke din Faders gamle S oster i Huset ? En stor <b>Aand</b> maa kun <b>vise</b> sig i sced-

<h4>248</h4>eller til Regler ? Taber Barmen efter Aar og Dag , ligesom en Verel , sin Kraft ; er Meget og mange Ting ikte saa yndige at see paa , ikke saa blode og kjodfulde at fole paa , hvad saa ! Hvad har da ogsaa et Legetoi at betyde ved en saa hellig , saa cervcerdig Forretning , hvor det ikke gaacr ud paa noget Ringere end at fkabe Mennesker og at efterligne Gud ! Romerne tillode sig aldeles formeligt cg lovligt at inddele Menneskene i Personer og Ting ; men intet Menneske er en Ting , og det er endogsaa liret , at vor ofte retfcerdige Sprogbrng anvender Artiklen det foran <b>Fruentimmer</b> . Qvin den , Kon en . Q v i n d e ! det herligste Ndtryk , som det Hcedersord Wgtef celle ikke kan sammenlignes med ! <b>Fruentimmer</b> burde vcere MQ6riß tNiuiiuni . Ere <b>Qvinderne</b> ikke Veen af vort Veen , Kjod af vort Kjsd , og , — hvad der er meer end Alt — <b>Aand</b> af vor <b>Aand</b> ? De kunne saalidt ester Godtbefindende skalte og valte med deres Person og Personlighed , at en saadan Kontract iiißo er ugyldig . Tjenester , forsaavidt som de ikke angribe Personligheden , kan Mennesket forcrre bort , scelge , leie ud ; kun ikke sit Jeg , og ikke det mindste Stykke af Fingeren , ikke et mindre Stykke af sit Legeme , om det saa syntes nok saa übetydeligt ! Det tilhorer ikke ham , men den hele Slcegt ! Vi ere Et , vi ere Guds ! — Mennesket er aldrig Middel , han er altid <b>Maal</b> ; aldrig Instrument , men den , som spiller ; han kan aldrig nydes , men er den Nydende ! I Mgteskabet forbinde to Personer sig , for gjensidig at nyde hinanden ; <b>Qvinden</b> onfker at vcere en Ting for Manden , og LEgtemanden forpligter sig paa sin Side i bedste Retsform ogsaa til at hengive sig som saadan . Da Begge nedlade sig til at vcere Instrumenter , paa hvilte der spilles verelviis , saa gaacr Nul op i Nul ; og denne eneste Meuneskenydelses- Contract er tilladt , nodvendig , guddommelig-viis . Men hvis nu denne Contract kun var sluttet paa visse Aar ? Manden mister ved LEgteskabet intet Ribbeen , hvis han er maadeholden , vedbliver han ofte ligetil sin hoie Alder at vccre en Mand i Mgteskabsforftand ; Mgtehustruen derimod bliver ved den forste bedste Leilighed saa ilde medtaget , iscer naar hun er en Blondine , at hun ncesten ikke er til at gjenkjende . — Ganske rigtig , kjcere Mgtemand ! men gaa hen i Herrnhuternes LEgteskabschor , og Du vil sinde , at

<h4>273</h4>ZEgtemanden . Hvad kan De sige , naadige Frue , som har udtastet et ordentligt System for den <b>qvindelige</b> Troskabs Grcrndser og synes at have sat denne Sag <b>Maal</b> og Hegn ?  Min Mand skeier ud ; og hvorfor siulde jeg " — ? Hvorfor? fordi De er en <b>Kone</b> og han en Mand . Troer De , at De bringer deres Mand paa rette Vei ved at undvige ham ? Hvis De er lykkelig nok til at bringe deres Mand , der begynder at komme paa Afveie , tilbage i LEgteskabssporet , saa fortjener De herved storre Beundring , end paa hun Tid , da De knyttede ham til Dem ved ham endnu übetjendte Mdigheder , De styrede hans frie Valg saaledes, at det maatte falde paa Dem . Den VErestrands , som De ifolge deres « eg te Kydskhed bar paa deres Bryllupsdag , tilkommer Dem nu for Deres ceg te Forstand . Anvend de Koketterier , som De helliger deres Tilbedere , paa deres Mand og betcenk selv , at det ikke er Dem nogen LEre at have Tilbedere , men vel en Mand . En udsogt Paaklcedning og en bestandig Neenlighed bidrage meget til at gjore en Mand contractmcessig fast , eller , uden juridisk Udtryk , gjsre ham bestandig . Hvidt er Uskyldighedens Symbol; Reenlighed kunde blive Symbol for Kydstheden ; en reenlig Paaklcedning tyder paa en kydst <b>Kone</b> ; Ureenlighed er et Skildt for Venus Pan dem os . Men <b>Qvinderne</b> forssmme disse Opmcrrksomheder , ligesom de , hvis Religionspligt det er at vadfke sig , hvilket Isder og Tyrker bevise . Maaskee bidrager Skjocget hertil ; thi stjondt Polakkcrue kun cultivere en lille Strimmel Stjceg , saa synes dog en <b>vis</b> forholdmcessig Ureenlighed at vcere deres Feil . — Det er vanskeligt , at holde en Mand stadig i Kjcerlighed og Troskab , og ( for ai tale figurligt ) ved et anderledes anbragt Baand at overraske Den , som har feet alle mulige Klceder ; men intet er lettere end at commandere et heelt Regiment Tilbedere; hvem vilde ikke lade sig hverve til en <b>qvindelig</b> Fane , hvor man tjener saa godt og sikkert ? hvem vilde ikke more sig , naar han ikke behover at betale Gildet ? Tidt bringer en <b>Kone</b> sin Mand saa vidt , at han selv forer Tilbedere til hende , og fratager ham Noglerne for saameget deslettere at kunne stjcele . — Men er De , Madame ! imidlertid ved denne euß6 6 ntr66 sikker paa , at deres Elskede ikke om kort eller lang Tid skriver sine Bekjendelser , ligesom Hans Jacob ,

<h4>314</h4>Mre og den Skam , den Lykke og den Ulykke at vcere deres Lige underkastet . Vore Allernaadigste ere ligesaavel Mennesker, som deres allerunderdanigste Tjenere ; og paa samme Maade er ogsaa Manden , som i Almindelighed staaer tilhsire i Huset ligesaa lidt fri for Feil som hans Mgtefcelle ; men desuagtet kan denne ikke gjsre noget klogere end at skikke sig i Tiden , faalcenge den onde Tid varer . Offre ikke ogsaa Mcendene med oiensynlig Fordeel Staten en god Deel af deres Menneskerettigheder ? og hvem kan svomme imod Strommen ? — Mcendene taale for Statens Skyld saa mange Uretfccrdigheder , at Ovinderne gjstre vel i med Nolighed i deres Huse at mode mindre Onder . Naar de vedblive at vcere <b>Qvinder</b> , formaae de Alt ved Vlidhed og Taalmod , saa at det i <b>Aand</b> og Sandhed kan siges om dem , naar de ere svage , ere de stcerke . Mcendene komme aldrig til <b>Maalet</b> i deres Kald ved Hjcelp af Taalmod og Blidhed ; det skulle de ikke heller ; thi netov fordi de ere stcerke , vaaligger det dem kun at seire ved Mod ; men tillige netop fordi de ere stcerke , burde de forjage den Tanke at ville herske i Huset . Hvem forlanger vel , at Koneu stal herske over Manden ? Hnu skal blot heller ikke beherskes af ham . Kunne da ikke to Mennesker leve sammen , som lade sig lede og fore af selvgivne Loves hellige <b>Aand</b> , uden at den ene af dem lofter sit stolte Hoved og , i Kraft af indbildt Magt , forandrer Tingenes Orden , og hellere forstyrrer deune end felger den ? Hvor underligt , at Mennesker , som regjere sig selv saa slet , saa inderlig gjerne spille Mester over andre ! stjsndt deres Sikkerhet » og deres No aftage i samme Grad , som deres Magtfuldkommcnheds Grcendser udvide sig . Paa Sa tur ns Tider var der hverken Herre eller Tjener , hverken Livegenskab for Person eller Ting ; hvor der er Frygt , er ingen Kjcerlighed ; thi Frygteu driver Kjcerligheden nd . — Det Forslag , at Manden skulde vccre Regent og Fruen Prein ierminister , er et Meglingsforslag ; men er det raadeligt , offentlig at tilstaae , at man treenger til en Premierministcr ? — Jeg maa frabede mig enhver Sammenligning og vil ndscette Sagen til Paakjendelse ved Lov og Dom . — Mceudene have Tilladelse til at trodse ; O.vinderne maae gjore Forestillinger ; Mcendene kunne paastaae ; Ovinderne formene ; naar alle Strceuge springe , kunne Mcendene lee ;

<h4>322</h4>skjelmsk at foreholde dem det . Vil man hore dristige Domme over regjerende Herrer , over deres Legeme eller deres <b>Sjel</b> , over Landets syv <b>Vise</b> og over dets andre Hundrede Forstandere ; saa bor man gaae til en opvakt Qvindes Orakel , og hun vil fcelde sin Dom og kjende for Net med samme Frihed , som hun vil belcegge det med Grunde . — Paa dette Kjon hviler Nevolutionens Haand . — Voltaire og Ro us se au gik i dette Kjons Skole . Den Tauke :  at Akademikere vare Videuskabernes Munke , Literatureus og de frie Kunsters Munke , " er en Qvindes Tanke . En stor Deel as de Galanterier , i hvilke <b>Qvinderne</b> indvikle sig , opstaae ikke af Tilboielighed og Kjerlighed , men af Lyst til at herste ; — de <b>vise</b> , at de trods al Undertrykkelse ikke kunne bringes dybt nok ned til at blive villielose , og holde sig skadeslose ved den 3 Ere , offentlig at trekke om med Konger og Fyrster , Ministere og Viismend , Geistlige og Digtere , og gjere ved deres Praris alle Theorier tilstamme , iudtil det Bieblik kommer , da de offentlig vife , hvad de af Naturen ere ! Man lese Historien , og man vil finde , at , fkjondt <b>Qvinderne</b> ikke regjerede , blev dog Alt regjeret ved dem , og at de forstode at hjelpe sig igjennem alle Vanskeligheder , for paa denne eller hun Maade at naae til <b>Maalet</b> . T h e - mist o kles tog ikke i Betcenkning at tilstaae , at hans Son rcgjerede hele Athen , idet nemlig hans Gemalinde vilde , hvad hendes Son vilde , og han , Themistokles , hv ^ ad hans Gemalinde vilde . — De mandhaftige Romerinders Herredømme over de qvindagtige Romere kan her tjene til Veviis ; og — bleve ikke de meest fordervede Mend langt mere nindskrenket beherskede af deres dem lignende Elskerinder , end brave Mend af deres verdige AZgtefeller ? stjondt man hos Romerne kun tilstod <b>Qvinderne</b> saare faa Nettigheder; netop derfor fynes niig , vare <b>Qvinderne</b> faa mcrgtige — og netop derfor ere de det efter Tid , Sted og Leilighed endnu den Dag i Dag . — Cesonia og Drufilla havde langt mere Magt over Cal i gula , og Mesfalina og Agrippina over Claudius , end den elskverdige A grip - pina over Germanicus . Naar man kun vil lade det smukke Kjon komme tilorde , saa vil man ved dets overveiende Kjcorlighed til det Gode og ved dets Verdenspatriotisme vinde mere end det selv . Allerede i det Smaa viser sig

<h4>372</h4>som man i daglig Tale siger , hverken Fugl eller Fisk , hverken kogt eller stegt ; han er paa intet Sted ; en mcmdhaftig <b>Qvinde</b> derimod er blot ikke paa sit Sted . Der kan ikke gives noget ssrgeligere Syn for en Fader end allerede i den forste Ungdom at opdage Prover herpaa . Selv Modrene ere misfornoiede med saadanne Sonner , fordi de efter deres Mands Dod ville soge Bestyttelse hos dem , og dog kuns kunne vente Lidt eller Intet . Er Faderen ikke ganske <b>vis</b> paa , hvordan det har sig med Ssnnen , saa gjor han meget vel i at bringe sin Son omtrent i det syvende Aar til et Gjerde , hvor han baade kan stige over og krybe igjennem < ; ( det forstaaer sig , at han ikke maa <b>vise</b> ham Veien , men lade ham selv vcelge ; ) stiger han over , saa bliver han en Mand ; men kryber han igjennem , saa maa Faderen beklage , at det er hans Son , og gjsr bedst i at lade ham blive Linnedvcever eller Skrceder , eftersom ifolge et bekjendt engelsk Ordsprog , ni Skrcedere udgjore kun een Mand . — Tillige , hoistcerede Ven ! gad Du nok vide , om din begavede Son vil plukke Laurbcer i Prosa eller i Vers ? Recipe : En Flåste Medicin , hver Time tredsindstyve Draaber i en Drik efter Behag . Lad ham selv maale dette punktlige <b>Maal</b> ; drypper han stadigt , saa er han en ' vingelam Prosaist ; lader han lobe , og tceller han under denne Plaskregn en to tre ligetil tredsindstyve , saa er han en Poet ; kan han strar spise med denne Skee , hvori han har talt de tredsindstyve, saa kunde han , hvis hans Forlcegger og han onske det , skrive mLtlwcla matksmatiea , ; men kan han i sire og tyve Timer ikke taale at see denne Skee , saa er han en lyrisk Digter ; kan han i sex Dage ikke uden kold Gysen bruge en saadan Skee — saa opdrag ham dertil , og , hvis Lykken er god , bliver han en Homer .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081903030'target=_>Borring, L. S., 1860, Fransk-dansk Haand-Ordbog</a><h4>540</h4>HlatelnellLinent , » ci . moderligt , som en <b>Moder</b> . spa a ; mus . Hlaterni36l- , v. n . slcegte sin <b>Moder</b> Hlaternite , / . Moderskab , det at verre <b>Moder</b> . sscetter af Maste . zl3.teur , ? « . ( Hlar . ) Mastemager , Ind > Nn , tliLMlltieieu , m . Mathemcitiker . Hlatkematic ^ uL , « . mathematisk ; strengt beviist , fuldkomment viS ; ... « . / . Mathe » matik ; iscer / ii . onBei ^ nei ' 1 « 8 - 8 , under » <b>vise</b> i Mathematik ; « tucliei- leB - 8 , studere Mathematiki leB - 8 pures , den rene Mathematik ; 1 « 8 - 8 unxteB snu hellere : lip ^ liciu6eB ) , den anvendte Mathematik . Hlailisiukticiueinom , aci . i mathematisk Forstand , ester Mathematikcns Regler . Nlltidi-e , / . Materie ; det Legemlige ; Stof ; Fe / . Emne ; Aarsag , Anledning ; B ' elever l » . U ' 6 eBBUB 6 s In . - , hcrve sig op over det Sandselige ; / « . stre enloncie c12 , N8 la » , la , loiiue euloncee llauB Ia - , have kun syndige Tilboieligheder , el . have en indskrcenket Forstand , et tykt Hoved ; traitei- k lancl uno - , behandle et Emne med Grun » dighed ; preter , lournir - u , lire , yde Stof til Latter ; il a 6 oune - d , e « cliBcourB , hau har givet Anledning til denne Tale ; tadle cleB - 3 , Indholds » liste i en Bog ; ( Hlun . ) - 6 ' ur , cl'g,r-ssent, det til Udmyntning smeltede Guld el . Solv ; ( Neci . ) Materie i et Såar ; Udtommelse , Excrementer ( iscer i d . Betydn . ) ; la - « lo Ia trnnBZ ) iration , Sved ; - me6io2.lL , <b>Kundskab</b> til Lcege » midlerne og Maaden , hvorpaa de til » beredes el . anvendes ; ( ^ ur . ) enkelte Dele af Retsvidenskabeu ( iscrr / ii . ) ; ... en - cle , ioc . / , / , . hvad angaaer , med Hensyn til ; en - cle religion , i Reli » gionsanliggender . N3 , till , m . Gaardhund , Hyrde » el . Slagterhund ; Kjoter ; slet el . kjed » somm ^ ligt Menneske ( / ' » / ' ) ; voilil un deau - B ' ii vc > ulg , it inai- ^ ie , dette Men » neske kunde gjerne udrette Noget , naai han kun vilde ; c ^ ui n , dun voiBin , s , dun m2.tin , hvo der har en god Nabo , kan verre tryg . Hlatin , ? » . Morgen ; Morgenstund ; Formiddag ; Z . une l ^ cury 6 u - , i , 6 eux lieul-e8 6 u - , Kl . 1 , Kl . 2 om Natten ; il trkvaille tout le - , han arbeider hele Formiddagen ; cc - , i Formiddags ; un 6 e ceB - 8 , en af disse Dage ; un deau - , engang , en Dag . forend mai :

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009091503005'target=_>Melbye, M., 1863, Fortællinger af Kirkehistorien</a><h4>1730</h4>eftersom Gud ( hed det ) jo dog stulde prises af alle Folk og forstod alle Eprog . Men da de tydske Keisere i det 14 de Aarhundrede tillige fik Kongemagten i Vohmcn , strebte de at kue saavel den folkelige som den kirkelige Selvstendighet» i dette Land , og snart blev ogsaa her latins ! Messe og i det Hele tåget pavelig Kittestik almindelig herskende. Dog fik Bohmeme under det tydske Herredomme en egen Wrkebistop i deres Hovedstad Prag og en Hoistole sammesteds ( 1348 ) ; og just dette blev et Middel til at vekke og udvikle mange af Folkets slumrende Krefter og kalde dem til Kamp mod Tydstemes aandelige Herredomme . Man trengte nemlig blot til Ledere i en faadcm Kamp ; thi hvor et Folk som her i over 3 Aarhundreder har baade HM og sjunget sit Modersmaal i Kirken , der lader det sig ille uddrive , uden at sinte Tilstugt i Husene , saa da Vohmem med den latinste Messe fik tydst Prediken i alle Kirkei . holdtes der Gudstjeneste i Modersmaalet omkring i Husene . Ja i Slutningen af det 14 de Aarhundrede W Folkets Vevidsthed om dets naturlige Ret vnngnet sna vidt . at 2 Folkevenner gruntede Bethlchems- Kapellet i Prag , hvor der altid stulde vere to behmiske Predikanter . Til en af disse valgtes 1402 Johan Huss , en kraftig , from og veltalende Mand , der mcd al Flid ivrede mod alt det fremmede baade romerske og tydste Vesen og saaledes snart blev Bohmerncs Moling og fremkaldte en stor folkelig og kirkelig Bevegelse . Det er naturligt , at en saadan Mand maatte for Folket blive som en Guds Engel ; thi nåar et Folk , der i Kirken har veret sit Modersmaal beredet, herer det levende lyde paany , da er det , som vi i et fremmed Land , hvor vort Liv staaer paa Spil og Ingen forstaaer os , <b>moder</b> en venlig <b>Sjel</b> , der taler baade Landets og vort Tungemaal . Hvilkensomhclst <b>Lerdom</b> en veltalende Mand vilde fore , maatte under stige Omstendigheder nesten nedvendig blive en Folkesag ; thi det er umuligt at sige et undertrykt Folk noget Ondt om sine Undertrykkere , som de ei stulde troe ; og har de mistet deres Frihet » , fordi

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014061108162'target=_>Smiles, Samuel, 1876, Karakteren</a><h4>220</h4>„ En god <b>Moder</b> " , siger George Herbert , „ er lige faa meget vcerd som hundrede Skolemestre . I Hjemmet er hun „ alle Hjcerters Magnet , alle Djnes Nordstjerne " . Hun er en stadig Gjenstand sor Efterlignelss hun er , som Bacon siger , „ en Verden af Lcerdomme . " Men Eksemplet er langt mere end Lceren . Det er Undervisning ved Handlinger . Det er en Lcere uden Ord , og det giver bedre Leveregler , end Tungen formaar at udtale . Ligs over for et slet Ekfempel gjor Lceren kun ringe Nytte ; Eksemplet folges da , ikke Lceren . Ja , en Lcere , som staar i Modstrid med Handlingerne, er vcerre end unyttig , da den blot bidrager til at <b>vise</b> den usleste og laveste af alle Laster Hykleriet . Selv Born forståa godt at bedomme Modsigelser , og en <b>Lcerdom</b> fra en Fader , der siger st men gjor et andet , skal snart blive gjennemfiuet af Barnet . Prcedikenen af Munken , som med en stjaalen Gaas under sin Munkekappe prcedikede om Wrlighed og Retskaffenhed , var ikke meget vcerd .

<h4>321</h4>Vi have omtalt Washingtons <b>Moder</b> som en i Forretningssager vel bevandret <b>Kvinde</b> . Et faadant Forretningstalent er ikke alene foreneligt med cegte <b>Kvindelighed</b> , men til en <b>vis</b> Grad en vcesenlig Betingelse for en velstyret Families Hygge og Velvcere . Forretningssceduaner have ikke alene med Hand els affyrer at gjore , men vedwre alle i Livet forsfaldende praktiske Sager - de komme i Betragtning overalt , hvor der er noget at ordne , organisere , ftrge for eller udrette . Og i alle disse Henseender er Bestyrelsen af en Familie og af en Husholdning lige saa meget en Forretningssag fom Bestyrelsen af en Butik eller et Bankkontor. Det krcever Methode , Nojagtighed , Organisation , Flid , Sparsommelighed , Orden , Takt , <b>Kundskab</b> og Evne til at tilpasse Midlerne efter Djemedet . I alt dette ligger Forretningsdygtighedens Boesen , og derfor maa <b>Kvinder</b> , som

<h4>336</h4>Det har aldrig vceret Bestemmelfen med <b>Kvinden</b> , at hun enten skulde vcere et tankeløst Veerwj eller blot en Pynt og Prydelse for Manden i hans Fritid . Hun eksisterer for fin egen Skyld faa vel fom for andres , og de alvorlige og ansvarsfulde Pligter , hun er kaldet til at opfylde i Livet , Kceve oaade en udviklet Forstand og et deltagende Hjcerte . Hendes hojeste Opgave skal ikke beståa i at erhverve sig enMasse unyttige <b>Kundskaber</b> eller vcerdilojeFcerdigheder , thi omendsijMt forstjellige Fuldkommenheder kunne forhoje Ungdommens og Skønhedens Ynde og Tiltrcekning , fom i sig selv er stcerk nok , ville disse dog <b>vise</b> sig at vEre-til forholdsvis ringe Nytte i det daglige Livs Forretninger .

<h4>341</h4>I sin Almindelighed kan det vistnok siges , at den Opdragelse og <b>Uddannelse</b> , som i de tidlige Aar passer for det ene Kjsn ogfaa passer for det andet , og at den Opdragelse og <b>Dannelse</b> , som opfylder en Mands Sjcel , ogfaa vil <b>vise</b> sig at vcere lige faa sund for <b>Kvinden</b> . I Virkeligheden tals alle de Grunde , som man har gjort gjceldende for Mandens højere <b>Uddannelse</b> , med samme Styrke for <b>Kvindens</b> Uddannelfe . I alle huslige Anliggender vi ! Husmoderens Virksomhed i og Betydning for Hjemmet kun kunne vinde . Den vil forpge hendes Omtanke og Forsorg , scette hende i Stand til godt forberedt at nmde de forskjellige Vanskeligheder , der kunne indtrceffe i Livet , finde paa bedre Methoder i Husets Styrelse , kort sagt i enhver Henseende styrke hend ? . I en tilkcempet « andelig Styrke vil hun finde en virksommere og sikrere Beskyttelse mod Bedrag og mod at ftres bag Lyset , end i en uskyldig og umistcenksom Uvidenhed . Ved moralsk og religiøs <b>Dannelse</b> vil hun sikre sig Kilderne til Indflydelfe , hvilke ere mcegtigere og varigere end legemlig Ande , og ved Tillid til sig felv vil hun opdage de sandeste Kilder til huslig Tilfredshed og Lykke .

<h4>1327</h4>denne . Ejendommelig for <b>Kvinden</b> er ogfaa en <b>vis</b> Livlighed og Beredthed til at handle , et skarpere Indblik i Karakteren samt et klarere Blik og en storre Lethed i Maaden at trcede op paa . Ide selskabelige Detailler r / ber sig hos <b>Kvinden</b> ligesom en medfpdt storre Opfindsomhed og Dygtighed , og af den Grund modtage Mandfolk med gode Manerer sedvanlig deres bedste <b>Dannelse</b> i Omgang med dannede og forstandige <b>Kvinder</b> .

<h4>1599</h4>Tidligere sagde man gjcerne om <b>Kvinden</b> , at det Hovedkrav , hun havde paa Beundring , laa i hendes Svaghed og Afhængighed. „ Skulde vi opstille et Billede af mandig Vcerdighed " , siger Sir Richard Steele , „ da maatts vi nedlcegge i dette <b>Visdom</b> og Mod , som Egenskaber vcefenlige for Manddommens Karakter . Ville vi derimod give et Billede af cegte <b>Kvindelighed</b> i dennes bedste Forstand , maa vi i dette lcegge blid Mildhed , om Frygt , og alle de Egenskaber , som adskille <b>Kvinden</b> fra det andet KjM , i Forbindelse med en <b>vis</b> Underordnen sig dette , men denne paa en saadan Mande , at Elstvcerdigheden derved ikke lider noget Skaar . " Folgelig vilde det snarere vcere hendes Svaghed end hendes Styrke , hendes Enfold niere end hendes <b>Visdom</b> , der maatte plejes . Hun skulde vcere bestemt ti ! at vcere et svagt , frygtsomt , taarerigt , underordnet Vcefen , kun med saa megen Forstand , at hun kunde forståa de intetsigende Behageligheder , fom henvendtes til hende af det „ stcerksre KM . " Hun skulde opdrages som et prydende Appendiks til Manden , mere end som et uafhcengigt Fornnftvcesen eller fom <b>Hustru</b> , <b>Moder</b> , Selskaberinds eller Veninde .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111903022'target=_>Wergeland, Henrik, 1859, Udvalgte skrifter</a><h4>13791</h4>Fortellingen om Morianernc blev afbrudt ved at Budcjen , det fsrste hun kom ind med Melken , raabte :  men Isssu Namn , <b>Mor</b> , Grauten er sviji . " Ingen af dem , som havde hsrt paa Skolemesteren , angrede paa at Grsden var sveden . <b>Mor</b> gav et Stykke stegt Flest tilbedste . som Skolemesteren fik for sin egen Mund , da han begyndte Maaltidet med den Bemerkning , at Trynetyrken plejede at anrette Kristenkjsdct , som altid maatte vcere fedt , aldeles som Svinesteg . Man kunde kjende Landet alt langtfra paa Lugten o . s . v. Den fsrste Hentydning var nok til at betage de andre Appetitcn ; men Skolemesteren var baade en stark <b>Aand</b> og en sterk Mave . laften sik han sidde i Hsjsedet, og befestede derfra endnu mere sin Anseelse ved nogle forferdelige Vedrifter af  Puntekorven " , som fra en liden Dreng af blev til Konge og stog Bonaparten. Disse havde han fra en liden i Kristiania udkommcn Vog , men sin anden <b>Lcerdom</b> havde han fra Studeringer i sine yngre Dage . om hvis Adtlsftighcd kun Den kunde gjere sig et Begreb , som havde seet Veger , der vare stsrre end Bibelen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016063008096'target=_>Luther, Martin, 1831, christelig Postille</a><h4>822</h4>Christus lader Sig sinde , men kun i Templet , kun i Det , som Guds er ? Men hvad er Guds ? Ere ikke alle Creature det ? Vel sandt . Alt er Guds ; men egentligen og i Besynderlighed er den hellige Skrift og Hans Ord det , thi alt det Andet horer naturligen til denne Verden , er af Naturen os givet . Hovedsummen heraf er da : Ingen stal understaae sig at soge nogen Trost , uden i Guds Ord , thi Sonnen sinder du ingenvegne , uden i Templet . Betragt Moderen ! Hun førstaaer det endnu ikke , hun veed ikke , at hun stal soge Ham i Templet ; og fordi hun soger Ham blandt Slagtnings og Kyndinge og ei paa det rette Sted , sinder hun Ham ikke . Derfor har jeg ofte sagt og siger det atter , at Intet stal prædikes i Christenheden , uden Guds rene og uforfalskede Ord . Hermed stemmer ogsaa dette Evangelium : de finde ikke Herren blandt Slcegtninge og Kyndinge . Og vi Christne gjorde ret , om vi gjorde dette Erempel til et almindeligt Ordsprog , som kunde bruges mod al <b>Lærdom</b> , der ikke er Guds Ord . Vi finde ogsaa paa flere Steder i Evangelierne saadanne Ez-empler og Lignelser, som <b>vise</b> , at man Intet stal lære , uden Guds Ord , og heller ingen anden <b>Lærdom</b> annamme , fordi man ikke finder Christum , undtagen i Skriften . Saaledes lcrse vi i Evangeliet paa Juledag , hvorlunde Eligelen , som forkynder Christi Fsdsel , siger til Hyrderne : Og det skal være Eder et Tegn , I skulle finde et Barn svobt , liggende i en Krybbe ! Hvorfor giver han dem ikke Maria og Joseph til et Tegn , men nawner kun og Krybben ? Grunden er den : Gud vil ikke <b>vise</b> os til nogen Helgen , ei engang til Herrens <b>Moder</b> , thi de kunne alle feile ; derfor maa Han angive os et sikkert Sted , hvor Christus ligger : det er Krybbel ! . Der sinder man Ham

<h4>879</h4>" hvad haver jeg med dig at gjore ? " Saa gjor Christus her : Han vil ikte gjore Guds Gjerning for Han stal , stjondt Hans egen <b>Moder</b> beder Ham derom . Fader og <b>Moder</b> ere ogsaa styldige , ja bestikkede af Gud dertil , at de ei efter eget Tykke og egen Andagt stnlle lære og fore deres Born til Gud , som ogsaa St . Paulus siger : I Fcrdre ! opirrer ikke Eders Born , men opfoder dem i Tugt og Herrens Formaning ! det er : lærer dem Gnds Bud og Ord , som I have lært , ikke Eders eget . Saaledes seer du ogsaa her i Evangeliet , at Maria viser Tjenerne fra sig til Christum ; thi hun siger ikke : Gjorer hvad jeg siger ! men : Hvad Han siger , det gjorer ! Kuns til Hans Ord stal man <b>vise</b> Alle og Enhver , dersom man vil <b>vise</b> dem den rette Vei . Dette Marias Ord : Hvad Han siger , det gjorer ! er og stal vare et dagligt Ord i hele Christenheden , hvormed al Mennestelærdom, Alt , som ikke er Christi Ord , stal omstodes , og det stal være hver mellig Christens stadige Troe , at hvad der bydes udenfor og over Guds Ord , det kan aldrig vare Christi Kirkes Bud ; thi Maria siger : Hvad Han siger , det , det , ja det gjorer , og ellers Intet . Og dermed have vi ogsaa Nok . Her seer du da tillige , at Troen ikke bestjcemmes , og at Gud ikke forlader den , men giver overfiodigere og herligere, end man beder . Thi her gives ikke alene Viin , men god og kostelig Viin og det i overstodigt <b>Maal</b> . Saaledes opmuntrer Jesus os til trostigen at troe paa Ham , om Han end tover med Hjælpen . Han er sanddrue og kan ikke fornægte Sig Selv ; god og naadig er Han , det siger Han om Sig Selv , og det beviser Han ogsaa , med mindre mail hindrer Ham og ikke henstiller Tid , Sted og Maade til

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009071303012'target=_>, 1863, Bibelen, eller Den hellige Skrift</a><h4>74</h4>Ezechias lude Kouges Mend have oversat ( samlet ) . Kap . 30 , I staar der den Overskrift : „ Agurs , lakes Sons Ord " ; K . 31 , I„Lamuel Konges Ord " . Slutningen danner K . 31 , 10 ff . , et afrundet Helt , det alfabetiske Stytte , som skildrer den dydige <b>Hustru</b> . Om denne Samlings Beskaffenhet » lader folgende sig formode : efter den ovenanforte Etterretning var Salomo Forfatter af et stort Ordsprogverk, hvoraf tun en liden Del kan vere gaact over i vor Bog » Mange Steder i den <b>vise</b> , at det Hele vanskelig kan hidrore fra een Forfatter : enkelte Ordsprog gientage sig , adstilligt Sammenhorende er adskilt ; der tales saaledes om Kongen , som denne selv vanskelig kunde have gjort , der forekommer Erfaringer fra det simple huslige Liv , som maatte vere Salomo fremmede . Nu forekommer i Samlingens Overskrift selv . . Ezechias Mend " , som have samlet en Del af Ordsprogene » Derfor synes det ikte unaturligt at antage , at ligesom der gjennem Kongernes Historie gil en Rekke af Profeter , ligesom der fra Davids . Tider og til Esra gaves Sangerssoler , og de fra disse udgangne Psalmer bleve benevnede med Stifterens ( Asafs ) Navn , har der ogsae veret en Kreds af <b>Vise</b> omkring Kong Salomo , der med ham er bleven Ophavsmend til Ordsprogsdigtningen i Israel ; og disse have da ogsaa nnder de efterfolgende Konger dannet en Forsamling , som beskjeftigede sig med at udbrede hellig Livsvisdom iblandt Folket (omtalt 22 , 17 og 24 , 23 ) . Under ugudelige Konger som Achab forsalet denne Anstalt , under fromme som Ezechias blev den gjenoprettet- Saaledes skulde man da fra K . 10 — 29 have en OrdsprogsamlinF , fra denne Skole af <b>Vise</b> ; deres Kjerne dannedes af egte salomoniste Ordsprog , uden at der om noget enkelt kan siges at det har Kongen til Forfatter . K . I — 9 var en Indledning , hvorved den hovedsagcliO salomoniste Samling blev indfort blandt Folket . At imidlertid Samlingens hele Indhold ikke gik meget udenfor den salomoniste Tid fremgaar klarligen af den Omstendighet » , at der ikke forekommer nogen Advarsel mod Afgudstjeneften og alle de Farer , som fremgaa af Forbindelsen med det afgudiste Vesen , i dem . Om de enkelte Dele og Anhangene vil der blive talt i Indledningerne til dem . Vor Bog indeholder Skjonheder , som blive skjulte for dem , som ikke sinder andet i den , end en usammenhengende Samling af Folkeordsprog. Vi saa allerede ovenfor , at denne Forestilling om Bogen er aldeles falss og af den Grnnd skadelig for Opfattelsen , at enhver Eftertenken over Sammenhengen herved iforveien bliver aldeles opgiven . Meget mere viser Bogen os hvad der efter Dommernes og den forste Kongetids Kampe ved indtraadt Rolighed dels var bleven virket ved den guddommelige Lov , d ? ls hvad der ved Fremadstriden pna den samme Vei skulde virkes ved Folkets <b>Vise</b> under guddommelig Oplysning . Saa los unegtelig Sammenhengen end er i et stort Antal Ordsprog , saa gaar der dog en Traad igjennem dem ^ som knytter dem til hverandre og holder det Hele sammen med de ovrige sildigere Boger og Apokryferne med Hentydning til N . T . ; denne er Leren om den guddommelige <b>Visdom</b> . Hvad i Psalmerne Modsetningen

<h4>3239</h4>i Bedre er en Fattig , som vandrer j sin Ustraffelighed , end en Mand mcd forvendte Lebcr og derhos en Daare . Det andet Led synes ikke at svare til det forste , idet man deri som Modsetning venter Tale om Rigdom . Men dette Slags Sammenstillinger , hvori det andet Led — uden Skade for Kortheden — videre udforer en anden Side af Tanken , er netop egen for den hebraiske Ordsprogdigtning . Saaledes stilles i V . 22 Logneren ved Siden af den Fattige . Ved « en Mand med forvendte Leber " maa her forstaaes en > der ved saadanne Kunster er bleven rig . 2 Ogsaa nnar der ingen Indsigt er , gaar det <b>Sjelen</b> ilde , og den , som haster mcd Fodderne , gaar feil . Med Ordet . . ogsaa " slutter dette Sprog sig til den Tanke : at den Dovne Intet kan vinde , at vedholdende Flid , Skyndfomhed er nodvendig for at naa <b>Maalet</b> . En Undtagelse fra denne Regel opstiller dette Sprog . Det synes , som om det for har stuttet sig til et andet , der nu er tabt , thi til V . 1 slutter det sig ikke . 3 Mennestets Taarskab forderver hans Vei ; men paa Herren vredes hans Hjerte . Begge Versets Led ere rettede mod dem , som legge Skylden for sin egen Daarstab paa Gud . 8 Den , som crhvcrver sig Forstand , elsker sin <b>Sjel</b> . « Erhverve sig Forstand " eller . . besidde " den bruges her i samme Betydning som vi sige : « en Mand af Hjerte ; " man erhverver sig det , idet man lever i Samfundet med Gud , der uerer og befordrer alle Mennestets hoiere Sjeleevner . io Vellevnet anstaar Me en Daare , hvormcgct mindre en Tjener at herske over Fyrster , en Utilborlighed , der under Livets Udvikling udstettes ved den guddommelige Verdensstyrelse . ii Gaa Overtredelse forbi d . e . tilgive , ikke hevne den . 13 Idel Ulykte for fin Fader er en daarlig Son , og et idcligt Tagdryp ere en <b>Kvindes</b> Kjevlcricr . Smlgn . 27 , 15. En daarlig

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011062123019'target=_>Sargent, Epes, 1880, Negerslaven Peculiar</a><h4>1220</h4>Det kan synes utroligt for Eder , som lMer hjemme i de nordlige Stater , at en Mand , som havde sin Forstands fulde Brug , skulde kunne sig skyldig i stige Afskycligheder . Men Mr . Onslow kjender Sågen tilbunds . Jeg beht « ver ikke at fortcrlle ham , at Slaveriet oplcrrer sine Haandlangere — Mcrnd , som endog er i Vesiddelse af en <b>vis</b> <b>Dannelse</b> — til Handlinger , der er endnu vcrrre , endnu mer djcruclske . Tro ikke , at den Tyran , som uden Skrupler bruger Svsben paa et sort Mcnneskes Nyg , vilde bctcrnke sig paa , at gj > ) re det samme med en Hvid . Mangen Overscers Dagbog vil <b>vise</b> , at over Trcdiedclcn af de Ulykkclige, som udscrttcs for denne barbariske Straf , er <b>Kuinder</b> ^ .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009090403011'target=_>, 1862, Vor Herres og Frelsers Jesu Christi nye Testamente, tilligemed Psalmernes Bog</a><h4>3893</h4>Overtredelser , og min Synd er altid for mig , Ps . 32 , 5. Jer . 3 , 12. 13. 6. For dig , « ja » for dig alene haver jeg syndet og gjort det Onde for dine Oine , paa det du stal uere retferdig , nåar du taler , vere ren , nåar du dommer . Nom . 3,4 ; 8,4 . Job 34 , 10. ? . Se . jeg er fodt i Misgjerning, og min <b>Moder</b> haver undfangct mig i Synd . Joh . 3. 6. Gph . 2. 3. 8. Se , du haver Lyst til Sandhed i inderste i Hjerte » , og du lader mig forståa <b>Visdom</b> i Lendom . Job 38,26 . Col . 1.26 3. Nens mig af Synd med Isop , faa bliver jeg ren ; to mig , saa bliver jeg hvitere end Sne . 3 Mos . 14 , 6. 4 Mos . 18,6 . 10. Lad mig hore Fryd og Glede , at de Ben maa fryde sig , son , du haver sonderstodt Ap . G . 16.14 . il . Skjul dit Ansifft fra mine Synder , og udstet alle mine Misgjerninger . Ps . 90 , 8. 12. Gud ! stab i miff et rent Hjerte , og forny en stadig <b>Aand</b> inden i mig . G.ech . 11,18 . 13. Bortkast mig itte fra dit Ansigt , og tag itte din Hellig- Aano fra mig . Viso . 8,4 . Eph . 4 , 20. 14. Giv mig igjen din Sa » ligheds Frud , ug en frivillig <b>Aand</b> opholde mig ! Nom . 8,15 . 45 , Saa vil jeg lere Overtredere dine Veie , og Syndere stulle omvende sig til dig . Luc . 22 , 22. 16. Fri mig fra Blodskyld . Gud . min Saiigheds Gud !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008110603012'target=_>Hansen, Maurits, 1857, Mauritz Hansens Noveller og Fortællinger</a><h4>1887</h4>netop afgjeres ved Salget af Hus , Tomt , Mpbler og Klcrder . Modig opfylder den gode Datter saa aldeles sine Pligter imod Forccldrenes Navn og Rygte , at hun Dagen ester at Moderen var bragt til Hvilestedet, ikke ejede andet end sine Klceder , en Bylt Linned , et Par Daler og — o hvor meget ! sin fromme Tro paa de Uskyldiges Gud og den sijpnne <b>Bevidsthed</b> , at have opfyldt sine Kjcrrlighedspligter i fuldt <b>Maal</b> imod de hedengangne Foraldre . I Norge lever , hun veed ikke ret hvor , en Fcetter af hende , en velstaaende Haandverksmand af en bekjendt agtvccrdig Karakteer . Til ham besiutter hun nu at henvende sig , tåger sit Pas , hilser for sidste Gang det Hus , hvor hendes Livs ftrste Akter udspilledes, den Gravhgj , der bedcrkker hendes omme Forcrldre , den Gade , — det Torv — den By — la , jeg ser hende , den modige , men saare bedrgvede unge <b>Pige</b> , hvorledes hun med den uvante Byrde over Skulderen vandrer med jevne Trin uden at se til hojre eller venstre hen over Landevejen , den Landevej , der stal fore hende ud i det vide , übekjendte Liv . Ester faa Timers Vandring paakommer hende en Frygt , som det hidtil aldrig havde faldet hende ind at ncrre , det er Frygten for , at hun ikke kan udholde at gaa . Den spcedlemmede Ungmy var vistnok vant til at bruge sine raste Arme og Hcrnder , men ej til at gaa hele Mile i Trcrk . Lotte tcenkte ikke paa , hvor snart Vanen vilde give hende Fcrrdighed i at tilbagelcrgge sine Dagsreiser ; Frygten for en stedse foreget Trcethed og Afmattclse bctog hende snart Modet . En Morgenstund sad hun aldeles modles paa en Strn ved Landevejen . Da rullede en Vogn langsomt forbi og standsede ved et Tilfccldc faa Skrirt fra Lotte . Hun skyndte sig hen til Skydskarlen og bad ham om Plads for sin Bylt indtil ncrste Skifte . Den Nejsende var netop min Hr. Lejtnant ; han giver sig i Samtale med den smukke Vandrende og tilbyder hende Plads i fin Vogn , Navnet Laholm er ham ikke übekjendt , og han sial sikkert i Kristiania faa opspurgt hendes Fatter .

<h4>2104</h4>Jeg lovede imidlertid saa meget hellere at hilse paa den gamle Frsken, som jeg fslte en <b>vis</b> « hjemlig og derved pirrende Sky for dette Menneske , som jeg hidtil alene kjendte fra min Moders Fortellinger . Disse bragte mig almindelig i en Stemning , som den , hvori adsiillige af Kruses Noveller scette Lcrseren , og da nu Lejligheden uventet tilbpd sig for mig til at gjpre dette maaske kun i mine Djne interessante <b>Fruentimmers</b> Vekjendtsiab optog Tanken om hin Visit i den Grad Pladsen i min Hjerne , at jeg glemte , at min Nejse havde et andet og vigtigere <b>Maal</b> .

<h4>2521</h4>Min <b>Moder</b> havde snart meddelt sine faa Vegivenheders Rcekke , saa meget desto snarere , da hun ikke var synderlig oplagt til ftaa dette Sted og med sit <b>Maal</b> for Rejsen at dvcele saa lcenge ved sine egne Livspartier . — Nu var da Raden til luliane , som uden at vente paa Opfordring , gjentog med en noget sterre Udforlighed , hvad min <b>Moder</b> i mere end et Aar vidste af Prestens Brev . Jo lcrngere hun talte , desto mere blev det hele klart for Tilhererinden . Et <b>Fruentimmer</b> af denne smackre , aldeles ikke stcrrke Bygning skulde med Vold have styrtet den fyldige , undersatte Ursula ud igjennem et trangt Vindu ? Og derncrst denne fine , fra enhver plump Handling og Tanke lige fravendt Sjcel skulde tilfredsstille sin Harme , sit Had eller hvilken som helst Lidenskab ved en saa udvurtcs-grov Misgjerning ? — Men derimod dette kongelige Aasyn , dette Glisabeth-Blik , denne i Vredens Bjeblik alt nedtrykkende Tale ; vistnok vred Ursula sig fornedret og senderreven som Ormen i Stev . Tanning viloe fradomme hende Mod ; hun behevede det ikke mere ; med en sterre Glendighed , end Dsdssmerte , havde Julianes kolde Hojhed gjennemtrcrngt hendes Hjerte . Viljelos gik hun bort , udvist ved Julianes Blik fra Livet og Lyset . — Det gik min <b>Moder</b> , som det saa ofte gaar os Mennesker ; vi kunne have feet og vidst noget i hele Aar , og dog trcrder bet os fmst paa een Gang i en <b>vis</b> Gjennemsigtlghed for Bjct . Det er det samme og dog noget ganske nyt . Ved at

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011070708225'target=_>Baardsen, Gjest, 1876, Gjest Baardsen Sogndalsfjærens levnetsløb</a><h4>118</h4>ham , fit jeg nok et godt Sengerum . leg tang . En <b>Pige</b> , som laa alene i en Seng i Arresten , begynbte at stamme paa dem og sagde , at de var nogle uforjlammede Slyngler , som saaledes bar sig ad med et ftakkels ungt Mennefie . ~ Han stal dog ikke ligge paa Gulvet . Kom hid du Gutten min ! " sagde hun i en blid Tone til mig . Det var i denne Arrest , saavelfom i de fleste andre , intet andet end Laster og Udyder at lcere ; man talede den hele Dag lun om Skjelmsstykker og Bedragerier , List og Underfundighed , Tyveri og allehaande Daarskaber . og hvo der havde vceret den flinkeste i at udfore deslige Streger blev holdt for den gjcrveste . Der var i denne Arrest ogsaa tyende Personer , der var rene Atheister , og ved deres forargelige Tale til enhver Tid ssgte at bibringe de Ulykkelige, der var indespcerret med dem , de samme Grundscetninger, de selv nceredc . De udlo enhver religios Ceremoni , talede om Guddommen med Haan og Foragt , negtede aldeles <b>Sjelens</b> Udodelighed og Haab om et bedre Liv . Vel fcestede denne stjcendige <b>Lcerdom</b> ikke Rsdoer hos alle , som bcfandt sig i deres Samkvem , og dette var , Gud ste Tak ! heller ikke Tilfceldet med mig . De gode Grundscetninger , min gode <b>Moder</b> og mine Religionslcerere havde givet mig , beheldt ftedfe Overhaand , og tilintetgjorde ethvert Forssg , man gjorde paa at faa mig til Proselyt , derimod lykkedes det dem alt for vel med nogle , og jeg har i den senere Tid havt et sorgeligt Bevis terraa med tyende Perfoner , som var i deres Samkvem og den Gang Ynglinger , hvilke nu ncere netop de samme Grundscetninger og med Haaronakkethed forsvare dem . Omtrent 12 Uger var forlovne , da min Dom , 2 Aars Tugthus-Nrbeide , faldt for Stiftsoverretten i Bergen , og kjed af tangere at sidde i Arrest erklcerede jeg mig tilfreds med Dommen og blev indbragt paa Bergens Tugthus , saavidt jeg erindrer , sicst i Juli Maaned .

<h4>994</h4>blamerede dig idelig og sogte ved de nedrigste Midler , ved List og Rcenker at lede mig i Fristelse og opnaa sine skjendige Hensigter . Endsijsnt Uslingen forfeilede sit <b>Maal</b> , og jeg afviste ham med den dybeste Foragt , blev jeg dog ikke fri for hans Overhceng , eller for at maatte hore idelig Bagvastelse om dig . Du ved , min Ven ! jeg elstede dig og agtede dig for heit til at hore med Ligegyldighed , at du blev bagvastet og belgiet af en Slyngel , du saa ofte havde overveldet med Godt , ja , som havde dig at takke for de Klcedcr han bar paa Kroppen . For at undgaa disse Übehageligheder , besluttede jeg at drage med min <b>Moder</b> ud paa Landet , hvor jeg har levet sardeles godt , og vilde levet bedre , om ikke Mindei om dig og din ulykkelige Skjebne daglig havde fyldt min <b>Sjel</b> med msrke ForestiUinger . Ak min Ven ! min bedste , min eneste Ven paa lorden ! du vil neppe tro , neppe forestille dig , hvormange Suk , der opsteg fra dette Bryst , hvormange bitre Taarer disse Dine har foeldet , siden du drog bort . O ! hvor kunde det voere anderledes? Nogle har sagt mig , at du var saa haardt fcengstet , at du aldrig mere kunde komme los igjen , og at du mgensinde mere kom hid ; andre , at du havde forlovet dig med den <b>Pige</b> , som blev arresteret med dig i Mandal , og var med hende reist til Sverige , og atter andre har paastaaet , at du var druknet ved at svsmme over et Vand , og flere deslige Historier — og du ved , at jeg for Mistankens Skyld ikke direkte har tunnet efterspsrge hos de Personer , af hvilke jeg kunde have faaet den rette Sammenhceng at vide , og har faaledes maattet laane Dre til hvad endel af dem , du staar i Forbindelse med , har foresnakket mig , hvoraf jeg dog ikke har kunnet komme paa det Rene , men svcevet i en martrcnde Uvished , i uafbrudie Bekymringer, i morke Anelser . Ligesom Tine nu havde tilkjendegivet mig de vigtigste Begivenheder , som havde msdt hende , medens jeg var fravcerende, saaledes fortalte jeg ogsaa hende i fuld Sammenhceng, hvorledes det var gaaet mig , hvad Lceseren allerede er underrettet om , uden at fordslge nogen Ting for hende . Jeg vidste nok , sagde Tine , at du ei var falsk imod mig , ei vilde , sluffe mig , jeg har havt for mange Beviser paa det modsatte ; men hvad de svrige Historier angik , da

<h4>1126</h4>mm Seng ; jeg mcerkede det imidlertid straz , reiste mig ufortsvet i Sengen og Mede med lav Stemme min tiofaste <b>Pige</b> . Tine besvarede min Hilsen med Blidhed ; hun kunde godt se ned i Arresten , fslgelig ogsaa betragle de frygtelige Lcenker , jeg var belcesset med. Hun gjss ved dette Syn , Graaden dcempede ncesten hendes Stemme , og hendes hede Taarer regnede mig uophsrligt i Ansigtet fra den Me Aabning i Taget , hvorigjennem hun talede . Vi trsstede hinanden saa godt vi formaaede , gav hinanden vaany de Lsfter at vcere bestandige under alle modende Tilfcelde, og aftalte videre , hvorledes vi for Eftertiden stulde mage det for at vezle Breve og undertiden paa samme Maave som nu at kunne tales ved , for at meddele hinanden Trost og Raad . Vi overlagde tillige , hoorledes vi under de tilstundende Forhsr siulde forklare os . Omtrent en Time var saaledes henrunden , da Tine sagde , at hun nu ei vovede lcengere at forblive hos mig , saasom hun befrygte.de, at Arrestforvareren , hvem hun vidste tog sin Middagslur paa den Tid , da hun gik , nu formodedes at vcere opstaaen igjen eller dog ncerved at staa op , og saaledes lettelig kunde have et eller andet Wrinde ind i Arresten , hvor hun var og lettelig komme til at observere hendes Fravcerelse. Jeg ventede ogsaa snart den anden Vagtkarl til « dage , og vi siiltes da for denne Gang under gjensidige Taarer og meo det hsitidclige Lsfte , aldrig at svige hinanden og aldrig at lade noget Middel blive übenyttet , der kunde lede til de : cngang foresatte <b>Maal</b> . Haabet , siger en <b>vis</b> Forfatter , er en Bi ; den suger Honning af enhoer Gjenstand , og bringer den til det menneflelige Hjerte . Saaledes var det ogsaa med mig denne Gang . Saa morke Udstgterne end syntes at vcere , havde Samtalen med min Elsiede dog fremkaldt et Glimt af Haab i min <b>Sjel</b> . Jeg havde nemlig den Uosigt , at hvis jeg kunde faa overtalt en af de Vagthavende , den som jeg havde stsrste Fortrolighed til , til at siaffe mig hensigtsmcessige Apparater , med hvilke jeg kunde paa en sublil Maade affijcere mine Jern , og ligeledes arbeide et forborgent Hul i Taget , medens jeg af ham bevogtedes , vilde jeg . naar andre lon ! paa Vagt , og som jeg vibfte gjerne lagde sig til at sove ! naar det led noget ud paa Natten , liste mig op

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013042608089'target=_>Bang, A. Chr., 1874, Hans Nielsen Hauge og hans samtid</a><h4>873</h4>omtrent paa samme Maade som Nutidens „ Bibelkvinder " ; men dette kunde blot lceres dem ved den historiske Udvikling . Til Bekrcrftelse paa , hvad her er sagt , stal jeg anfore Vidnesbyrd fra Samtiden . John Haugvaldstad skriver : „ Et Aar senere kom en <b>Pige</b> , fodt i Gudbrandsdalen , som og havde Opbyggelse hos os og flere Steder . Ved hende fik jeg megen Ledelse i <b>aandelig</b> Henseende " ' ) . I mine Samlinger har jeg Optegnelser af en gammel <b>Kone</b> , der skriver Folgende om sine Foraldre : „ Min <b>Moder</b> havde i lcengere Tid fornd , ja fra sin tidligste Barndom , havt et kraftigt Kald paa sig : da saa Rygtet kom ud blandt Folk , at En ved Navn Hans Nielsen Hauge reiste omkring i Landet og talede til Folket , de skulde omoende sig , erholde Syndernes Forladelse , tro og blive salig , bler , det deres ( 3 : Foreldrenes ) alvorlige Tanke ; men paa hvad Maade dette skulde ste , forstod de ilke , iscer min Fader . Der fandtes da ilke et Menneske blandt dem , som runde veilede dem . Te lcengtes da meget efter , at Nogen stulde komme og hjelpe dem tilrette . Herren saa deres Lengsel og tenkte paa dem med sin Naade . Der kom tvende unge <b>Piger</b> , som nylig vare opvakte og forte et kristeligt Liv , disse vare opvakte ved Hauges Virksomhed; dette blev dem lo : Foreldrene ) til stor Velsignelse / ' Hauge forteller , at da hans forste Skrifter „ kom for Lyset " , da var der Flere , der „ bekom Guds <b>Visdom</b> og Kraft til at bekjende lefu Navn fom en Frelfer fra al Synd " - ) Saaledccvar det ogfaa . Allerede ved Udgivelfen af andet Oplag a ' Bogen om „ Guds <b>Visdom</b> " kunde han tale om „ tro Hyrder " , som ei er „ Leiefvende " , faadanne , der „ ei for Penge myrder , eller bruger verdslig Magt " " ) . I Begyndelfen af 1797 treffer man hans Medarbeidere i og langs Veien til Elverum " ) ; i 1798 optroede de paa Kongsberg og Eker " » : i 1799 <b>moder</b> man dem paa Modum og i Hougsund ' ) , i Gudbrandsdalen

<h4>2840</h4>det fandt man sig dog i paa Grund af hans Neises Beskaffenhed og de ovrige Omstændigheder ; derimod tog man llgetil Anstod af hans „ udvidede Begreb " og hans mere lette Opfatning af flere Ting , at han f . Ex . barberede sig om Sondagen , at han endog reiste paa Herrens Dag , at han syslede med fin Strikning og stig „ Forfcengelighed " ' ) . Uagtet Alt dette var udvortes Ting , var det dog klart , at der ud fra det gamle Herrens Redstab traadte Vennerne imsde en fremmed <b>Aand</b> , dette var tydelig mcerkbart gjennem den „ friere Vandel i det Udvortes " . Nogen Indflydelse paa hans indre Liv havde maaste Samvcrret med Vennerne under denne Neise ; jeg siger maaste ; thi hans Skrivelser fra Hesten 1809 udviser omtrent den samme aandelige Forfatning som fsr , kanske dog med en Smule mere Inderlighed 2 ) . Brevene i Tidsrummet fra December 1809 til December 1810 derimod udvise paa en glcedelig Maade , hvorledes Hauge efterhaanden fandt tilbage til de gamle gode Stier . Saavidt man kan fe , var det iscrr gjennem 3 Omstændigheder , at Gud drog ham . I Begyndelsen af 1810 sit han Efterretning om sin Svoger Kjsbmand Loofes Dpd , og Tabet af den trofaste Ven og Medarbeider gik ham dybt til Hjerte , saa han undertiden af Graad ikke kunde tale . Hans Broder Mikkel med <b>Hustru</b> og Flere befsgte ham i denne Tid ; de vare sammen paa Toien og trssteoe hverandre med Guds Ords Trosta ) . Der spores et kjendeligtOpsving i hans aandelige Liv fra denne Tid , og Minderne fra Vcekkelfestiden blive mer og mer levende . Saaledes hedder det f . „ Salig var den Stund , Gud kaldte os ved sit Ord og <b>Aand</b> , og lyksalig er al den Tid , vi har med fri Villie anvendt alle de af Gud betroede Midler til vore og Medmenneskers Sjeles Foradling, eller givet det Gode Rum til at virke i vort Hjerte , og sogt at virke paa Andre igjen . Bliver vi nu i den sande Tro og Kjcerlighed med Retfcerd , som i <b>Visdom</b> over Godt , jo mere jo bedre , indtil vor Ende , ak hvor hei bliver da vor Lyksalighed'.

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112301014'target=_>Taine, Hippolyte, 1875, engelske Literaturs Historie</a><h4>86</h4>og kun i øieblikke strømme over sine Bredder , den vil hurtigt og af sig selv træde tilbage i sit tilvante Leie . Franskmanden er mild og al Naturen civiliseret , den stærke og plumpe Sandselighed har kun li6sn liltrNkninF for ham , han elsker den Ntii-usiiFO Samtale , hans I ^ inksH og gode Smag værge ham let mod alle Kneipe Manérer . Grammont har for megen <b>Aand</b> til at ynde vilde Drikkelag , thi de give , naar Alt kommer til Alt , ingen Glæde : at slaae Glas itu , at vræle , at sige Smudsigheder , at drikke til man faaer Ondt , i alt dette er der ikke Meget , der kan friste lidt finere Sandser . Grammont er født Epikuræer , og hverken Fraadser eller Drukkenboldt ; kan soger behagelig Tidsfordriv og hverken Wil6Blsß Glæde eller dyrisk Nydelse . Ganske vist er han ikke uden Dadel ; jeg vilde ikke betroe ham min Pung , han glemmer altfor let Forskjellen mellem Mit og Dit , og fremfor alt vilde jeg ikke betroe kam min <b>Kone</b> , thi paa det Punkt har han ikke særligt fine Fornemmelser ; hans gale Btr6F6r ved Spillebordet og ligeoverfor <b>Damer</b> smage lidt vel meget af Bedrageren og Forføreren . Og dog kar jeg Uret iat bruge disse store Ord , hvor der er Tale om ham ; de ere for vNFtiF6 , de knuse et Væsen saa fint og smukt som han . Disse tunge Æres og Skjændselsklædninger kunne kun bæres af alvorlige Folk , og der er ingen Ting , Grammont tåger alvorligt , hverken andre eller sig selv , hverken Lasten eller Dyden . At faae Tiden til at Faas tornmsii ^ t , det er alt , kvaH han bekymrer sig om ;  man kjedede sig ikke længere ved Armeen " siger Hamilton , „ siden han Bw6t6 til den ; " det er hans Ære og hans <b>Maal</b> , det er det Eneste , han arbeider for , det Eneste , kan tænker paa . Hans Tjener bestjæler

<h4>414</h4>sat å la négligence ; Mistress Stewart med sin skarpt ombøiede Hat , sin røde Fjer , sine blide Øine , sin romerske Næse oss sin 1 u166n6t6 Figur * ) . " Man kommer tildass6 til White Hall ,  <b>Damerne</b> gaae og komme , snakke , lege med deres Hatte og deres Fjer , bytte saa hver efter sin Tour , prøve de Andres og lee . " I Baa godt 86 i8kad kan Galanteriet ikke mangle .  Fra Paris kommer der hver Uge vellugtende Handsker , Lommespeile , rigt forsynede Etuier , Abrikosdeig , V6Uusst6N < i6 01161 - , og alle andre smaa Elskovsvarer;" London leverer derimod  de solidere Foræringer , jeg mener alt Sligt som ørenringe , Diamanter , Ædelstene og smukke 6 ulH.6uin66r * * ) . De Skjønne tåge Imo6 det , ganske som om det kom langveis fra * * * ) . " Intrigeme ki ^ HB6B , Gud maa vide 1 dvaH Tal og mod hvad <b>Maal</b> . Naturligvis gaaer ogsaa Samtalen sin Gang ; man taler NHit og omstændeligt nok om den stolte Miss Warmestre ' s Eventyr ;  hun havde aabenbart regnet Feil , hun blev overrasket og tog sig nu den Frihed at gjøre Barsel for selve Hoffets Øine . " Man hvidsker ganske sagte om Miss Hobarfs Forsøg , om Miss ChurchilFs lykkelige Ulykke , da hun , der var meget styg , havde <b>Aand</b> nok til at falde at Hesten og derved røre Hertugen af York ' s Øine og Hjerte . Eidder Grammont fortæller Kongen Historien om Tennes eller Almisseuddeleren Poussatin , og Alle forlade Dandsen for at høre paa ham ; som han slutter sin Fortælling lee de som skulle de briste . " Man seer , at vi her ere i den fine Verden , om end IKK6 i Ludvig den Fjortendes , og at det er den samme Strøm , om der end her er mere Skum . Ogsaa her er det Hovedsagen at more sig og at <b>vise</b> sig ; det er et stort <b>Maal</b> at blive en Mand paa Moden , man kan blive berømt ved en smagfuld Klædning : Grammtmt er helt fortvivlet , da hans Tjeners Tyveri tvang ham til

<h4>552</h4>er » MandenafÆre * ) « , som mandengang sagde , forExempel PMlinte , Ariste , Clitandre , Eraste * * ) ; det er det Eneste , der kan belære os samtidigt med at det morer os . Dets <b>Begavelse</b> er en kraftig og selvstændig Tænkning , men den er udviklet ved Selskabslivet ; dets Karakter er en grundfæstet Hæderlighed, men den er lempet efter Selskabslivet . Man kan efterligne en saadan Mand uden at forsynde sig mod Fornuft eller Pligt , han er hverken en Nar eller en Vellystning ; man kan etterligne ham uden at forsømme sine Interesser eller udsætte sig for at blive latterlig , han er hverken en Dumrian eller en Tølper . Han har <b>Kundskaber</b> , han forstaaer Trissotin ' s og Lycidas ' s umenneskelige Sprog , men bruger det kun til at hugge dem sønder og sammen , til at slaae dem med deres egne lis ^ lsr og more hele Galeriet W a deres Bekostning . Han taler endogsaa om Moral og Religion , men Baa naturligt og ligefremt , med Baa klare Beviser og en « åa oprigtig Varme , at han interesserer <b>Kvinderne</b> og faaer de ivrigste Selskabsmennesker til at høre paa sig . Han har Menneskekundskab og han udtaler den , men i saa korte Tankesprog , i Baa levende Billeder , med saa træffende Spot , at hans Filosofi er den bedste Adspredelse af alle . Han er tro mod Bin ødelagte Elskerinde og sin da ^ taits Ven , men i al Btilns6 og med Ynde ; alis hans Handlinger selv de bedste , forskjøimes ved at udføres let og som var det en Selvfølge , han gjør Intet u6sn Behag ; hans store Talent er Levemaade , lian anvender den ikke blot i Hverdagslivet , ved den daglige Omgang , men ved de vol6BoMßts Bammsnßts6 , midt under de største Foriegenheder. En fornem Slagsbroder beder ham om at være hans Sekundant ved en Duel , han tænker efter et Øieblik , un6Blaasr sig i nogle faa Sætninger , og uden at spille

<h4>1249</h4>Spøg og sin Høflighed . Til Dermis , som havde rost ham , skrev han : » De gode Egenskaber , De tillægger mig , ere ikke mere mine end Maanens Lys kan siges at være dens , siden den kun skinner med sin Broders Gjenskin * ) « . I 6 t Brev til sin Kusine indflettede han som en morsom lille Fortælling følgende Enkeltheder om en meget svær <b>Dame</b> , han havde reist sammen med : » Hendes Vægt gjorde at Hestene maatte trække meget haardt , men for at give dem Tid til at puste , 8 t1 , u68666 hun os ofte , og angav en eller anden naturlig Nodvendighed , og sagde , at vi vare alle Kjød og Blod . « Disse smukke Ting synes dengang at have moret <b>Damerne</b> . Hans Breve bestaae af plumpe formelle Hefligheder , tunge firkantede Komplimenter og mathematiske Buk ; selv hans letteste Tale bliver en Afhandling , han stiver de mindste Übetydeligheder af med regelmæssige Perioder . Til Grev lioLk6Bt6l- , der havde sagt ham en Kompliment , siger han : » Jeg føler , at det ikke sømmer sig for mig paa nogen Maade at kappes med Deres H6i ' liFli6d , der kan skrive bedre over det sletteste Stof , end jeg kan over det Bedste . « Dette fandt man , var et aandrigt Svar . Jeg har truifet smukke Steder hos ham , men aldrig et behageligt ; selv i sine Afhandlinger mangler han Smag . Personerne i hans Porsøg over Dramaet troe sig 6 n6nu xaa Bkoi6hl6nk6n , de <b>vise</b> deres <b>Lærdom</b> og anføre Paterculus , og det tilmed paa Latin , de bekjærape Modstanderens Definition og bemærke , at den er gjort a genere et fine , istedetfor som den gode Regel kræver efter Slægt og Art . I en Fortale siger han med lærd Værdighed : » Man bebreider mig , at jeg har valgt udsvævende Karakterer til Protagonister eller Hovedpersoner i mit Drama , at jeg har gjort dem lykkelige ved Stykkets Slutning hvad der strider mod Komediens Lov , der vil at Dyden skal belønnes og Lasten straffes * * ) « .

<h4>1544</h4>imidlertid see lidt nærmere paa denne baandbesatte Laps , der kurrer Florian ' s Sange i sin lysegrønne Dragt ; den selskabelige <b>Aand</b> , der har fert ham ud i disse Flauheder , har ogsaa fert ham andetsteds hen , thi i Frankrig i det Mindste er Samtalen en Jagt efter Tanker . Endnu den Dag i Dag , trods Nåtidens raistænksomme og sørgmodige Sæder , er det ved Bordet , medens Kaffen drikkes , at den heie Statskunst og den strænge Filosofi træde frem . At tænke , især at tænke hurtigt , er en Fest , det er som et Bal for Aanden , og den seger det da ogsaa med Iver og Lyst . Hele vor <b>Dannelse</b> skriver sig derfra . Ved Aarhundredets Begyndelse kunde man høre <b>Damer</b> under en flygtig Visit frem sætte skarpsindige Afhandlinger eller give udarbeidede Portrætter ; de forstaae Descartes , de finde Behag i Nicole , de give Bossuet deres Bifald . Snart be- FvuHs de smaa Aftengilder , og ved Desserten forhandler man om Guds Tilværelse . Høre Theologi og Moral , naar de behandles skjent og kraftigt eller livligt og træffende , maaske ikke med til Selskabssalens Ny deiser , og ere de ikke et af de Riges Smykker ? Det vittige Lune , Vid og Begeistring , finde der deres Plads , de bølge og funkle som en let Flamme over alle de Emner , hvormed de næres . Hvor kraftig var ikke det attende Aarhundredes aandelige Virksomhed . Har man nogensinde seet et Samfund begjærligere efter de høieste Sandheder , dristigere til at soge dem , hurtigere til at finde dem og fyrigere tiJ at tåge imod dem ? Disse duftende Marquiser , disse Lapse i Kniplinger, hele denne fine lille Verden , saa pyntet , elskovsrig og letsindig som den er , strømmer til Filosofien som til

<h4>2121</h4>Derefter igjen taler hau om Midlerne til at bringe Handlingen i Gang , og udvikler hvorledes de fonnes og under hvilke Betingelser , og hvorledes de anvendes og til hvad Nytte ; det er som 83,2 , man en 81166 k61 ' gaae en Trappe efter Fjæl for Fjæl , for at sikkre sig at den var bygget akkurat . Man maa ikke troe at det Kunstlede støder ham , tværtimod han beundrer det ; han fmder Gudfaders lange Taler og de hofmæssige Hefligheder , hvormed Tre-611i^k666U8 Personer glæde hverandre , ædle og ophøiede ; han kan ikke 866 iioF6t Falskt eller Noss6t Hdkkagsii ^ t i Miltons Engle , trods deres Leiere velser , trods deres Lign cd med Kapelmedlemmer og Soldater , trods deres lærde Stridigheder og skjøndt 66 tale som ivrige Puritanere eller blinde Tilhængere af Kongen ; han synes at Adams støvede <b>Lærdom</b> og hans Prædikener for sin <b>Hustru</b> svare aldeles til den rene Uskyldighedstilstand . Klassikerne fra 66 to sidste Aarhundreder have i Virkeligheden aldrig formaaet at opfatte den menneskelige <b>Aand</b> uden som civiliseret: Barnet og Kunstneren , den Vilde og den Begeistrede forstode de ikke , og da saa meget desto mindre Alt , hvad der skulde tænkes hoiere end Mennesket . Tilværelsen var for dem kun Jorden og Jorden igjen kun en Læsestue og en Selskabssal ; de kunde hverken naae Gud eller Naturen , og naar 66 prøvede derpaa , forvandlede

<h4>3283</h4>med en Regn af Tankesprog og nydelige Vendinger , medens han tilsøler sin Broders Rygte og forfører sin Naboes <b>Kone</b> . Da det følsomme Menneske først var bygget , fik det snart sit Heltedigt . En Skotlænder , en aandrig Mand , der havde altfor megen <b>Aand</b> , havde skrevet sn Bunke daarlige Vers for sin egen Regning , nu vilde han holde sig skadesløs , han drog om i sit Fædrelands Bjerge , ledte efter maleriske Billeder , samlede Stumper af gamle Sagn , tilsatte det Hele med en god Sum Veltalen- Ks6 og iikstorik , og lavs66 Baai66sß den keltiske Homer , Ossian , der med Oscar , Malvina og det hele Følge drog Europa rundt , og omtrent 1830 endte med at skaffe vore offenlige <b>Piger</b> og Haarskjærere Døbenavne . Macpherson viste sine Læsere en Efterligning af Naturtilstanden , der ikke var altfor sand , thi den virkelige yderlige Raahed vilde have været stødende , men som dog var saa godt holdt eller afskrevet , at den kunde danne en Modsætning til Nutidens <b>Dannelse</b> , og overbevise Læserne om , at de saae den rene Natur for deres Øine . Her var en levende Følelse for de store , kolde , mørke skotske Landskaber ; Regnen faldt over Høiene , Birken skjælvede i Vinden , der laa Taage over Himlen og en Übestemthed over Sjælen , som tillod enhver Drømmer her at gjenfinde Stemningerne fra sine egne ensomme Vandringer og sørgmodige Grublerier ; her var ridderlige Bedrifter og Ædelmod , Helte , der kjæmpe ene med en hel Hær , trofaste <b>Piger</b> , der døe paa deres Elskedes Grav , en lidenskabelig stærkt farvet Stil , der synes at være afbrudt , men i Virkeligheden er en Kunststil , der kunde fortrylle en af Rousseau ' s Elever ved sin Varme og sin Finhed ; her var i Sandhed nok til at henrive Tidens begeistrede Ungdom , disse civiliserede Vilde og dannede Naturelskere , der drømte om det vilde Livs Herlighed , medens de afrystede Pudderet , Frisøren havde efterladt paa deres Frakke .

<h4>3287</h4>mod den følsomme Overveielse ; de talrigste og mest læste Digte ere rørte Afhandlinger . Uden Anledning at henfalde til heistemte Betragtninger , er virkeligt det følsomme Menneskes Særkjende ; seer lian en Sky , drømmer hanstrax om det menneskelige Liv og laver en Frase , derfor seer man i dette Øieblik mellem Digterne en Vrimmel rørte Tæn- K616 og grædefærdige B ^ jsnaanH6r . Gray , den gnavne Eneboer fra Cambridge , og den ædle Tænker Akenside , der begge med stor <b>Lærdom</b> etterligne den græske Poesi , Beattie , den moralske Filosof , der har en ung <b>Kvindes</b> Nerver og en gammel <b>Piges</b> Luner , den elskværdige Goldsmith, der skrev den mest fortryllende af alle protestantiske Idyller , Landsbyprcesten i Wakefield , den stakkels Collins , en begeistret Yndling , der blev kjed af Livet og nu kun vil66 læse Bibelen , som blev gal , blev indespærret , og i sine frie Mellemrum flakkede om i Domkirken i Chichester og fulgte Musiken med sin Hulken og sin Klage ; Glover , Watts , Shenstone , Smart og endnu flere . Titlerne paa deres Værker <b>vise</b> tilstrækkeligt deres Karakter : En skriver et DiFt over Indbildningskraftens Glæder , en Anden Oder over Lidenskaberne og Friheden , En skriver en Elegi fra en Landsbykirkegaard og en Hymne til Modgangen , en Anden Vers over en ødelagt Landsby og over de omliggende Civilisationers Karakter , en Tredie et Slags Heltedigt over Thermopylæ og atter en Anden en ung MinstreVs moralske Historie . De ere næsten alle alvorlige NVQ6 , de troe M3 , det 16661168 Overvægt , de ere begeistrede for ædle Tanker , de have kristelige Følelser eller Overbevisninger , de tænke stadigt over Mennesket , de have Smag for tungsindige Betragtninger , for Beskrivelser og Paakaldelser , de blæse Liv i almene Begreber og bruge evige Allegorier , og naar de ville naae det Store , gaae de gjerne P33 , Stylter . En ak de mindst strænge OF mest berømte var Nattetankernes Forfatter , Young ; han var Geistlig OF Hofmand , han havde forgjæves prøvet paa at

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008102303002'target=_>, 1848, Evangelisk-christelig Psalmebog</a><h4>2142</h4>17. Borne-Psalme . Mel . Kommer til mig , sagde Guds Son . 5 « » O gode Gud ! jeg tatter dig . En lcrrllg <b>Moder</b> gav du mig , En god . en ttotaft Fader ! leg kjender ikke Livets Vei ; Men deres Raad og <b>Lcerdom</b> ei Paa Veien 2. O Gud ! lad mig betanke ret , Hvad fra mit fsrste Aandedrat Jeg deres Omhu skylder ; ^ Da rsres jeg sta inderlig , Og Kiarllghed til dem og dig Min hele <b>Sjel</b> opfylder . 3. Mit rsrte Hjerte bryder ud I Bsn nl dig , nlgode Gud ! Den Svage kan du styrke . Giv , jeg dem lyder redelig ! De lcere mig at kjende dig . Og trofast dig at dyrke ! 4. Lsnbu dem selv med Mistundhed . Og dm Velsignelse udbred Til hvad de foretage ; Og lad dem finde Trsst hos dig . Rnnr Alderdommen ncermer sig Med strsbelige Dage ! 5. Lad Dydens Glade , Hnnb og Roe I deres Hjerter altid boe ! Du stedse hos dem vare ! Og . nåar fra mig du kalder dem Til gode Sjetes rette Hjem . Lsn dem med evig 3 Ere !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006082200023'target=_>Bang, A. Chr., 1875, Hans Nielsen Hauge og hans Samtid</a><h4>2718</h4>det fandt man sig dog i paa Grund af hans Reifes Beskaffenhed og de ovrige Omstændigheder ; derimod tog man ligetil Anstod af hans „ udvidede Begreb " og hans mere lette Opfatning af flere Ting , at han f . Ex . barberede sig om Søndagen , at han endog reiste paa Herrens Dag , at han syslede med sin Strikning og flig „ Forfcengelighed " ' ) . Uagtet Alt dette var udvortes Ting , var det dog klart , at der ud fra det gamle Herrens Redstab traadte Vennerne imode en fremmed <b>Aand</b> , dette var tydelig mcerkbart gjennem den „ friere Vandel i det Udvortes " . Nogen Indflydelse paa hans indre Liv havde maaste Samvceret med Vennerne under denne Reise ; jeg siger maaste ; thi hans Skrivelser fra Hosten 1809 udviser omtrent den samme « åndelige Forfatning fom for , kanste dog med en Smule mere Brevene i Tidsrummet fra December 1809 til December 1610 derimod udvife paa en glcedelig Maade , hvorledes Hauge efterhaanden fandt tilbage til de gamle gode Stier . Saaviot man kan se , var det iscer gjennem 3 Omstændigheder , at Gud drog ham . I Begyndelsen af 1810 fik han Efterretning om sin Svoger Kjobmand Lo oses Dod , og Tabet af den trofaste Ven og Medarbeider gik ham dybt til Hjerte , faa han undertiden af Graad ikke kunde tale . Hans Broder Mikkel med <b>Hustru</b> og Flere besogte ham i denne Tid ; de vare fammen paa Toien og trostede hverandre med Guds Ords Der spores et kjendeligt Opsving i hans aandelige Liv fra denne Tid , og Minderne fra Voekkelsestiden blive mer og mer levende . Saaledes hedder det f . „ Salig var den Stund , Gud kaldte os ved sit Ord og <b>Aand</b> , og lyksalig er al delt Tid , vi har med fri Villie anvendt alle de af Gud betroede Midler til vore og Medmenneskers Sjeles Foradling, eller givet det Gode Rum til at virke i vort Hjerte , og sogt at virke paa Andre igjen . Bliver vi nu i den sande Tro og Kjcrrlighed med Netfcerd , som i <b>Visdom</b> over Godt , jo mere jo bedre , indtil vor Ende , ak hvor HK bliver da vor Lyksalighed".

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008062004080'target=_>, 1861, Læsebog i Modersmaalet for Skolernes højere Klasser</a><h4>5542</h4>at henvise til sin alvorlige Levesiik , medens han tillige raisonnerede over den komiske Skrivemaades Nytte . Men hint joviale Grundprceg var udholdende gjennem hans hele Levnetsloh : det viser sig ikke blot i Beretningen om hans Ungdoms Reiseliv , men ogsaa i den fremrykkede Alders Bekjendelser , som n ' aar han siger , at det bedste Middel mod den sygelige Tungsindighed , der kunde overkomme ham , ' var „ at lcese lystige Skrifter eller selv at skrive Skjemt . " Mellem Holbergs Sjelenatur og hans phy- We Tilstand var der en Stridighed , som han ofte omtaler og som undertiden bringer ham til den Itring at han er sig selv en Gaade . Til sin lyse ' og freidige <b>Sjel</b> havde han faaet en hypochondrist Legemsbefiaffenhed , der ncrredes ved de Tilvcennelser , den havde fremkaldt , og plagede ham med uvilkaarlige Wngstelser og Forstemmnger . Heraf fornemmelig kom Skyggesiderne og de ' tilsyneladende Modsigelser i hans Liv . Vi se ham i Orford mellem de Unge , yndet af Alle for sin Skjemt og Munterhed ; vi folge ham paa hans vidtlsftige Reise , hvor han uforfærdet bcerer Forladthed , Armod og Sygdom , vandrer paa sin Fod gjennem Landene fra Middelhavets til " Nordsoens Kyster , altid med et klart , betragtende Blik , altid saaledes deltagende i Livet , der omgiver ham , at vi fceste Tro til hans Ord , naar han siger at han udenlands „ mere har studeret Mennesker end Bibliotheker . " Men saa finde vi ham igjen , endnn i den kraftige Alder , ensom , ncesten utilgjoengelig i sin stillesiddende boglige Syssel , ofte , som han siger , i Grcemmelse over Bagateller , og saa pirrelig , „ at en Hund , som gjor , en Dor , som tillaases , kan bringe ham af SM . " Men stjont dette hans Hjemliv baade var betinget af og forsgede hans Hypochondri , indfalder dog ogsaa her et formildende Lys , og vi trceffe ytringer af hans livlige Natur . Hans Verker <b>vise</b> noksom , at han ogsaa hjemme har kunnet trcede ud af sin Ensomhed og vcere med i Samfundslivet , hvortil allerede hans Embedsstillinger og Eiendomme paa mange Maader knyttede ham . Ja , det var ham endog en kjer Adspredelse at scette sig hos <b>Damerne</b> og underholde sig med dem . Men iscer maa man loegge Mcerke til at Holbergs Liv var meget forstjelligt efter Aarstiden . Om Vintren var han helst aflukket fra Verden , da fslte han stcerkest sine Onder og var saa kuldstjer , at han nodig forlod Kakkelovnskrogen ; men om Sommeren havde han det bedre- For han fik Landgods gjorde han tilfods Udfarter i det Gronne ; men siden bessgte han sine Gaarde og tog sig af deres Drift og Opkolnst . Paa Landet forbedredes hans Helbred ; der syslede han i det Frie , plantede Trceer , og havde Oie med Skovens og Markernes Vert- „ Jeg finder Behag , " siger han , „ i at see Ardens Gront fremspire og indhostes , at see Kjor og Faar gaae ligesom i Procession til deres Samlingspladse Morgen og Aften . " Paa Landet var han selskabelig , bessgte Gudstjenesten/og indlod sig gjerne med Almuen . „ Aldrig , " bemcerker han etsteds i sine Epistler , „ taler jeg med - ' onder uden jeg Icerer noget af dem . " Ogsaa her var han den ttoge lagttager , og medens han modtog de levende Indtryk , samlede han overalt Stof til Betragtning og Eftertanke . Der er et Trcek i Holbergs strenge Livsbillede , hvorved det , ligesom ved et Smil , forfljonnes . Det er hans Kjcerlighed til Musik og hans musikalske <b>Dannelse</b> . Han spillede Violin og Floite , men havde iscer paa dette Instrument vundet Fcerdighed . I Oxford blev han , efter aflagt Psrve , optaget som Flsitespiller i et musikalsk Selskab , og da han paa sin tredie Udenlandsrejse vendte hjemad og kom febersyg til Amsterdam , blev hans Medvirken i en musicerende Vennekreds ham saa velgjorende , Feberen at ophorte . Ogsaa i sit Hjem bevarede han denne Interesse .

<h4>5545</h4>Han var Medlem af et harmonisk Selskab , og Musikens Tilstand i Kjobenhavn laae ham stedse meget paa Hjerte . Som Alumnus paa Borchs Kollegium havde han i en af sine Deklamationer handlet „ 66 clulo66in6 mußio6B , " og endnu i Slutningen af sit Liv var han opmcerksom paa Tonekunstens Udvikling ; men han fandt nu med Beklagelse , at en Modesmag havde berovet den „ Harmonie og naturlig Ssdhed . " " Saaledes skifter i Holbergs Levnet det Milde , Tiltalende med det Strenge og Frastodende , og hermed kan vi sammenligne Grundtrækkene i hans Charakter . Denne var i sit hele Anlceg selvstoendig , fast og udholdende. „ Hvad jeg engang har faaet i Hovedet , " siger han , „ har jeg aldrig godt kunnet faae ud igjen . " Det Haarde hen og det Prcrg af Selvgodhed , som ved hans Livskaar blev mere kjendeligt , havde allerede en Viodvcegt i hans <b>Sjels</b> naturlige Munterhed , og forædledes tillige ved den humane Verdensanffnelse , han havde tilegnet sig . Han manglede vel Gemyttets dybere Varme og havde desmere af den satiriske Aare , men der var i hans <b>Sjel</b> en sund Temperatur , som han , trods alle Anfægtelser udensra og hos sig selv , forstod i det Hele at bevare ; deraf kom det store Maadehold , ja , man kunde sige , det godmodige Troet i hans Satire , og hans Modbydelighed for Grovheden og „ Morositeten " i visse Satirikeres Skrivemaade . I sine fsrste Stridsskrifter viste han sig hvas og som En , der var let at tirre , men efterhaanden lcerte han sig Fordragelighed. Mod sine Übillige Modstandere , Gram og Rostgaar ' d , ncerede han ingen varig Uvillie , og med hvilken Skaansel behandlede han den forunderlige , ncergaaende Joachim Paulli , der omstobte hans politiske Kandcstober , og vilde gjore ham rigtig klog paa Komedieskrivning ! Denne lysere Grund r Holbergs Charakteranlceg har ogsaa i hans tidligere Aar skinnet tydeligt frem i hans Udvortes . " Han var smakker og ietbygget og havde et saa mildtformet Ansigt , at han undertiden blev anseet som en - forklcedt <b>Pige</b> . Dette sit ungdommelige , og , som han selv kaldte det , jomfrunalske Udseende bevarede han lcenqe , og endnu i hans Billede fra Oldingsalderen ser man de fine og blode Grundtrock . Det Voerste , der er sagt om Holberg , er Beskyldningen for Gjerrighed. Dette Omdomme var ham bekjendt ; han omtaler det ved mange Leiligheder og soger for sig selv og Andre at forklare hvorledes hans Liv : denne Henseende billigt skulde opfattes . „ Jeg haver stedse , " siger han , „ holdt Pengegjerrighed for en af de Feil , som mest vanhelde Mennesket," og han tilfsier , at alt hvad han har gjort for at undgaae ' den stemme Mistanke var paa Grund af det Skin , der laae i hans tarvelige Levemaade , blevet frugteslsst . Imidlertid raisonnerede han over Sagen og har ogsaa i en udforlig Betragtning , der udgjor det forste Stykke af hans moralske Tanker , baade vittigt og skarpt fremstillet den Gjerriges Lastvcerdighed . Holberg var fra Barndommen af vant til streng Sparsomhed, og maatte siden i en lang Aarroekke fore de smaa Kaars nsisomme Liv ; derncrst fordrede hans Helbredstilstand stedse , dersom hans Virkekraft siulde bevares , en streng Dicet og et regelbundet Liv . Vin var ham en Gift , og af Selskabsliv els almindelige Nydelser kom der kun Lidet paa hans Part . „ Hvis megen og lakker Spise , " siger han „ kunde eontribuere til min Helbred , tog jeg ikke ! Betænkning at give en Dukat for hvert Maaltid . " Hertil kom den ufravigelige Orden og Nsiagtighed , han overholdt i Smaat som i Stort , og hvorved hans tarvelige Levemaade end mere fik Udseende af pinlig Indskrænkning . Saaledes blev han den noieregnende Husholder , og naaede derved en Velstand , som dog ei havde vceret noget <b>Maal</b> for hans Strceben . Man har bebreidet

<h4>6721</h4>Partie , og denne Medfslelse meddelte sig til de Tilskuere , der satte sig ind i Brudens Stilling . „ Endnu , " sagde Frue H . , „ er hun kun bedrovet over , at hans Udvortes ej behager " hende ; herefter opdager hun nu Dag for Dag hans Feil ; da bliver " det vcerre . " Efter en Times Tid kom der Bud , at Brudgommen var i Kirken . En elegant med brogede Tepper prydet Slcede optog Bruden . Foran denne kjerte Hirsches Tjener Haver ( i daglig Tale Savka ) i en simplere Slcede med Obragoli i begge Hcender , og Blodet med Saltkarret paa Skjodet , til Kirken , hvorfor en anden Mand stod paa Mejerne og forte Tommerne ; efter Bruden kjsrtc vi med Hirsches Familie . I Kirken stod Brudgommen , ifort en luvslidt graae , slet passende , ilde staaren og uklcedelig Frak , med et. rodt hvidtcernet Torklcede surret om Halsen , et tort stramt Ansigt , smaae Ojne og sort Haar , der hang som Lysevceger ; kort han saae ud , ikke alene som en Tsmmermcmd , men som en Troemand . De fortes samlede frem , efterat man havde givet hver af sem et lidet med Guldblade forziret Vorlys i Haanden . Popen holdt en kort Messe med sin <b>vis</b> Konus og ovrige Sangere ; gik derpaa hen og tog deres Ringe af deres Fingre og " lagde dem paa Alteret . Derefter blev der igjen sunget og Bonner loeste ; endelig Ringene tilbagebragte fra Alteret og ombyttede. Tilsidst bleve to store Messingkloner frembaarne ; Popen lod Brud og Brudgom kysse et Sted hver paa sin Krone , hvor formodentlig et Crucifir var afbildet ; derpaa satte han dem Kronerne paa Hovedet , lagde deres hsjre Hcender sammen , og tagende dem om de sammenlagte Hcender , forte han dem under Vang tre Gange rundt om et lidet paa Gulvet staaende Altar . Endelig frembar Popen et Solv-Crucifir , hvilket Brud og Brudgom , samt Hirsch og Brudens <b>Moder</b> og alle Andre , som havde Deel i Hojtideligheden , kyssede . Kronerne bleve aftagne , og nu kjortc Brudgommen hjem til Sit med sin Brud , som han hidindtil neppe havde talt ti Ord med. Haver kjorte atter foran Toget med Odrasen , Brodet og Saltkarret , hvilke fra nu af vare Brudens Ejendom og en Helligdoms hvormed hun igjen kunde bestjenke en tilkommende Datter , naar ' denne skulde giftes . Ved det Aftensmaaltid , som man nu indtog i de Nygiftes Huus , vare alle Brudens Medtjenere fra Hirsches Huus indbudne , og Haver vartede op. De vare alle utilfredse med dette Partie , og vrede paa Svacha , som havde drevet det igjennem , iscrr Haver , som havde vceret indtaget i Bruden , men ej maatte faae hende , saasom han var Livegen , og en Livegen ej kan gifte sig , hverken naar , eller med hvem han vil ; men kuns , naar hans Herre eller Ejer befaler , og med den han befaler . Da Haver ombar Kagerne , bod han derfor Svacha dem allersidst , hvorover hun blev stodt , og sagde , at han fortjente et Orefigen. Efterat han var fcerdig med Opvartningen , gik han derfor hen til hende , og sagde , at hun havde budet ham et Orefigen ; det havde hun selv fortjent , og han vilde ej lcenger forholde hende det ; hvorpaa han gav hende en dygtig Kindhest , og hun drev bort i frygtelig Vrede . Hun var desuden ogsaa vred paa brudgommen , fordi han ej havde betalt hende rigelig nok for hendes ivrige Bestroebelser med at stemme Bruden til hans Fordeel . Et Par Dage efter bessgte den unge <b>Kone</b> Frue H . og da denne spurgte hende , hvorledes det gik hendes Mand ? svarede hun bedrovet : „ oh ! af ham bliver der aldrig Noget ; svacha er forbittret paa ham , fordi han har betalt bende for daarligt , og hun tager <b>Maalet</b> fra ham . " Hun troede , at Svacha , for at hevne sig " , havde forgjort ham , saa at der ej kunde komme Liv i ham , eller at han nogensinde kunde faae Ordet .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112100014'target=_>Baur, Wilhelm, 1880, Billeder fra evangeliske Præsters Husliv i Tydskland</a><h4>160</h4>« Er det ikke , » udraabte en ved denne Leilighed, « som om der til de 3 Hedningekonger ved Frelserens Vugge endnu kommer en fjerde , der ogsaa bærer sin Gave frem ? » Men Goethe selv udtrykte sig djærvere og raabte Gang efter Gang , idet Minderne fra Straszburg syntes paany at kaldes tillive : « Ak Børn , hvor vi ere dumme , hvor vi ere dumme ; vi indbilde os , at vor Le6Btemo < ser ikke ogsaa har været skjøn ! » Hvor skjøn vor Bedstemoder har været , ja derom kunde jeg have Lyst til at fortælle lidt , — ikke blot vor kjære kristne Bedstemoder siddende med den hvide Kappe og Brillerne foran Billedbibelen , den hun ofte væder med sine Taarer , idet hun viser den til sit Barnebarn , men ogsaa vor hedenske Stammoder , med det graa Håar omvundet af Hovedlinet og i Bin uldne med Peltsværk kantede Klædning , hvorledes hun med det dvoe Blik og det alvorsfulde Udtryk , lig en Norne fra en svunden Tid , forteller ved den blussende X ^ aminild den nye Slægt om Guder og Helte . En Mands sædelige <b>Dannelse</b> , — dette bør ogsaa prædikes for tyske Præstehuse — finder en fortræffelig Maalestok i den Maade , hvorpaa han stiller sig til <b>Kvinderne</b> , i det <b>Kvindelighedens</b> Ideal , som han opta ^ er i sig i 3 in anende længtende Ungdom , i hvad der drager ham Ken til den <b>Kvinde</b> , som han beder dele sin Livsskjæbne med sig , hvilken Tone han anslaar for sit huslige Liv og hvor høit han vurderer <b>Kvindernes</b> Indflydelse i det selskabelige Liv . Den samme Maalestok tør ogsaa anlægges paa Folkene ; kvor <b>Kvinderne</b> holdes i den skjændigste Trældom , der turde idetheletaget den største sædelige Forkommenhed raade , derimod ere de i sædelig Henseende ædleste Folkeslag ogsaa tilbøielige til at <b>vise</b> <b>Kvinderne</b> den sømmelige Ære . De tydske Stammer gaa deri foran alle andre . Man har

<h4>654</h4>Fra Doltzig kaldtes Aszmann til Gartz , derfra til Hagen i Forpommern , og begge Gange blev hans Kaldel3e varslet ham i en Drøm . I ( ^ art ? kavde han midt under Krigens Nød Glæden al en stor Opvækkelse inden sin Menighed ; ikke alene kom Bønnens <b>Aand</b> med Kraft over Børnene , men engang forelæste han under Præ-6ilcenen den 85 de Psalme , og det gjorde et saadant Indtryk , at den hele Menighed paa mere end tusinde Personer , Kvc » ridlan6t grevelige og adelige Standspersoner , faldt ned paa sine I ^ nse og under utallige Taarer offentlig bekjendte og begræd sine Synder . I Hagen opfyldtes en Forjættelse , som under 3 vZ6oininenB Dage var forkyndt ham ved en Røst , som uforandret lød 3 Gange : » Du skal endnu faa en Søn . « Hans <b>Hustru</b> fødte ham en Søn , 16 Aar efter sit sidste Barns Fødsel . For denne sin Søn har Faderen beskrevet sine Livsførelser og tilføiet dertil alle Slags nyttige Formaninger . Han formaner ham til at forblive en Jesu Kristi tro Tjener og hverken at agte paa Verdens Bifald eller de I ^ Nlde3 Vantro ; angaaende Theologiens Studium giver han ham nsiaFtiA Undervisning . » Han skal være tilfreds med et lidet Kald , hvori Guds <b>Visdom</b> maatte sætte ham , og fly de store Embeder , forat han i Stilhed kan bygge Guds Rige og desto mere uhindret drive sine Studier og efter Leilighed være sin Næste til Tjeneste . Fordi der i de sydlige Egne af det tydske Rige er Udsigt til at tunge Tider forestaa baade i det Aandelige og i det Timelige , saa gaar mit faderlige Raad ud paa , at han efter vor , hans Forældres , Død vil vælge sig de nordlige Egne til Opholdssted , hvor den gud-

<h4>674</h4>om Aftenen Glæden ved det behagelige Samvær i Haven , eller en rolig Spadsertur ned til Våndet , saa megen Omgang med Slægtninge og Venner som muligt , og om der i Nabolaget findes Mennesker af høiere social Stilling og <b>Dannelse</b> , da en gjensidig Paavirkning , som baade i det adelige Hus og i Præstehuset bevarer fra at rustificeres . Den 60 - aarige Provst i Berlin , som beskriver sit Liv , føler sig næsten altfor ærbar og værdig til at tale om dette sit første Præstelivs Vn6i ^ ne6er . Og ogsaa i Berlin har han det ved alt Arbeide dog saa behageligt , at den Tanke ikke falder ham ind : » O , hvor fjernt ligger Alt , hvad engang var mit ! « men endnu i sin Alderdom taler han om sin Ungdoms Kjærlighedslykke , fordi det vilde være utaknemmeligt mod Gud at tie derom . » Vore smaa Børn , vore Læsestunder , vor lille Have med dens yndige Udsigt ble ve os gjennem hyggelig , fortrolig , venskabelig Omgang en næsten uafbrudt Kilde til rolig Nydelse og Adspredelse . Saadanne übetydelige Omstændigheder , som Himlen maa bevare enhver egentlig Biografi fra at omtale , nedskriver jeg her ikke ugjerne , kun af den Grund , at det altid har været en af mine behageligste Følelser at mindes ethvert Gode , som jeg nar nydt i mit Liv , det være sig lidet eller stort , og at forhøie og fordoble Glæden over at erindre dem ved en taknemmelig Ihukommelse af deres første velgjørende Ophavsmand . I at vedligeholde og i altid livligere at opøve denne lykkelige Sindstilstand var mig fornemmelig ogsaa min dyrebare <b>Hustru</b> behjælpelig , hvis naturlige , muntre og ikke mindre fromme <b>Sjel</b> aldrig følte sig gladere , end naar vore Hjerter under Nydelsen al alle saadanne Livets Venageligneder og Nydelser kunde i Fællesskab hæve sig til ham , ved hvem de dleve os skjænkede . «

<h4>932</h4>beviste Hjertets Haardhed , saa synes de nu for Tiden mere at hidrøre fra dets Ynkelighed , derfra nemlig , at Folk fra Begyndelsen af ikke lægge an paa noget ordentligt med sit Liv og med sin Kjærlighed og ikke forbinde noget Begreb eller <b>Formaal</b> dermed . « Tidens geniale Ungdom , selv ikke bunden ved noget Ægteskabs Baand , følte sig derfor dybt krænket ved ædle <b>Kvinders</b> Skjæbne , som levede under slige uværdige Om- BtNn6iSne6er , anto ^ sig Ægteskabet og hævdede Individualitet6Nß Ret ligeoverfor den ydre Anordning , Hjertets Ret li ^ eoverfor det lcol6e 3 NdvanßrllNß3iFe og Bozte at skildre Kjærligheden , hvorledes den forstod at forsone og gjennemtrænge Legemligt og <b>Aandeligt</b> , Frihed og Bundethed . Friedrich Schlågel , som havde besmudset sin Ungdom og sine Aandsgaver ved et uordentligt og vildt Liv , førtes ved disse Bestræbelser for- at ophjælpe Ægteskabet til to store Udskeielser : den ene var hans Roman » Lucin6e « , et i æsthetisk og sædelig Henseende lige forfeilet Makværk , i Form som i Indhold lige letfærdigt og sønderrevet , den anden var hans Forbindelse med en gift <b>Dame</b> , Dorothea Veigt , født Mendelsohn . Og Schleiermacher , som aldrig til <b>Kvinder</b> stod i et andet Forhold end det ædle venskabelige , geraadede paa denne Tid i en dobbelt Forvildelse : han forsøgte at redde Schlågels Bog ved sine : » Fortrolige Lreve om Lucin6e « , idet han underlagde det liderlige Makværk de reneste Tanker , og hans Forsøg paa at redde en gift <b>Dame</b> , Eleonore Grlinow , udaf hendes ulykkelige Ægteskab , gik hos ham lidt efter lidt over til Ønsket om at hjemføre den Fraskilte som <b>Hustru</b> til sit eget ' Hus . Fra begge Forvildelser , de sørgeligste i hans Liv , blev han reddet gjennem bitre Smerter , , og Gud gav ham Lykke til at faa grunde et Hjem , et

<h4>1094</h4>Gjenstand for hans Overveielse , og lagde sig saa igjen ned . Snart derpaa indtraadte Dødens Forbud « , saa fortæller Kan3 Biograf ; » en Ven foresagde ham disse Apostelens Ord : » Jeg agter ogsaa i Sandhed alt for Tab imod Ypperligheden af <b>Kundskaben</b> om Kristus Jesus , min Herre , og agter det for Skarn at være , paa det at jeg kan vinde Kristum og befindes i ham som den , der ikke har Retfærdigheden ved Loven , men den , som kommer ved Troen paa X.ri3tum , at jeg kan kjende ham , hans Opstandelses Kraft og hans Lidelsers Samfund . « Hvilket han saa tilføiede : » og Herlig heden derefter . « Nu lød Miinsterens Klokke , som hver Fredag Kl . 1 1 forkynder det evige Offer paa Golgatha , og hvis høitidelige Toner , , hvorunder ogsaa hans <b>Moder</b> var hensovet , beredede kam en stille Høitid , saa ofte han hørte dem ; han gav sin trofaste <b>Hustru</b> det sidste Afskedskys , den korte Dødskamp indtraadte og man hørte ham sige disse Ord : » Han har alene maattet træde Vredens Oliepresse ! « Og under hine Klokketoner gik hans <b>Sjel</b> hjem . Hans Ansigt var i Døden overordentlig skjønt , en alvorlig Fred , en stille Majestæt hvilede derover og lod endnu mere end i Livet den høie <b>Aand</b> lyse frem- , som havde havt sin Bolig i dette Ler . «

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111903003'target=_>Levinsen, C., 1854, Real-Concordants, eller Bibel-Ordbog</a><h4>413</h4>13 29. 5 Mos . 7 , 1. los . 11 , 4 ; der var forberedt paa Krig , los . 17 , 16 ; stod under Konger , 4 Mos . 21 , 1. 5 Mos . 7 , 24 los . 10 3. 23. 11 , 1 flg . 12 , 9 fig. sml . Dom . 4 2 ; og havde allende ved Agerdyrkmng og Handel naaet en <b>vis</b> Grad af <b>Dannelse</b> og borqerliq Velstand i det af Naturen saa vel udstyrede Land , 2 Mos . 3 , 8. 17. 13 , 5 ; denqanq da Israeliterne under Moses og losva begyndte at angribe dem , 5 Mos . 6 , 10 flg . los 24 13. Derfor gik det langjomt med Landets Indtagelse , og Israeliterne maatte finde sia i at mange Cananiter vedbleve at boe i Landet , los . 16 , 10. 17 , 12. Dom . 1 , 27 fla 3 1 fla . ; stundom imod at de betalte Skat los . 17 , 13. Dom . 1 , 28. 30. 33.35 ; saa at de endnu holdt sig til Davids , og paa enkelte Steder indtil Salomos Tid , 1 Kong . 9 , 16 20 flg . Andre maatte drage bort , og navnlig stal en stor Deel virre kommen til Africa . ^ Israeliterne hadede Cananiterne baade som deres Fiender og som Afgudsdyrkere , 2 Mos . 23 , 24 flg . 34 , 13 ; alligevel var denne Fslelse lcenge mindre levende hos Folket og dets Herskere , og hvorvel det var forbudet Israeliterne at mdgaae Wgteskab med cananitiske <b>Qvinder</b> , 2 Nos . 34 , 16. 5 Mos . 7 , 3 ; overtraadtes dette Forbud oftere lige til Erilet , Dom . 3 , 6. 1 Kong . 11 , 1. 16 , 31. Esra 9 , 1 flg . ; og drog ofte Deeltagelse i cananitisk Avgudsdyrkelse med sia , . Propheterne ivrede derfor altid strengt imod Cananiterne, Ez . 16 , 3. Zeph . 1 , 11. Sach . 14 , 21. Oananites ( Ivreren ) er et Tilnavn for Apostelen Simon , Matth . 10 , 4. Mare . 3 , 18. Ordet betyder det samme som Zelotes , Ap .

<h4>1510</h4>smelte sammen . Ligesom hos andre Folk findes den fsrste Art tidligst hos Hebrcrerne og oprindelig i Forbindelse med Musik og stundom med Dands , Dom . 16 , 25. 1 Sam . 18 , 7. 2 Sam . 6 , 5 ; den knyttede sig til mccrkelige Nationalbegivenheder, vigtige ' Seire , Befrielse fra overhengende Fare , ' 2 Mos . 15 , 20 flg . 4 Mos . 21 , 27 flg . Dom . 5 , 1 ; og fik ifslge Guds Fsrelser med dette Folk tidlig sit bestemte Prceg som religws Poesi . I den fsrste Tid Fvedes den iscrr af <b>Qvinder</b> , 2 Mos . 15 , 20 flg . Dom . 5 , 1. 11 , 34. 1 Sam . 18 , 7 ; men sterre <b>Uddannelse</b> modtog den ved David, af hvem den paa en vcerdig Maade blev indfsrt i Helligdommen , hvor den bidrog meget til at hawe den offentlige Gudsdyrkelse . Efter hans Tid dyrkedes den af Propheter og Leviter , saaledes Korah ; og blomstrede endnu efter Exilet . At der ved Siden af den hellige Poesi ogsaa var en verdslig , sees af Es . 5 , 12. ler . 25 , 10. Amos 6 , 5. Sir . 49 , 2. Der findes saaledes deels Hymner til Gud , Ps . 8. 46. 47. 48. 66. 67. 75. 76. 78. 104. 105 ; og for Konger , Ps . 20 ; deels Klaqesange saavel med Hensyn til enkelte Personer , som til det hele Folk , 2 Sam . 1 , 17. Ps . 7. 56. 76. 79. sml . Begr . Sog . Psalmernes Bog er en Samling af saadanne Digte . Hsisangen er mere at betragte som et Kjccrli ^ hedsdigt . — Den didactifie Poesi knytter sig til Ordsproget ; den dyrkedes af Salomo, 1 Kong . 10 , 1 fig. ; efter hvem Salomos Ordsprog derfor har faaet Navn , skjsndt denne Bog ogsaa indeholder noget af andre Forfattere , sml . Ordspr . 30 1 , 31 , 1. Fredens Dage under Salomo vare vel egnede til stille Eftertanke . Denne Digtart oplivedes atter ester Exilet og Frugten deraf samledes i Jesu Sirachs Viisdom . Paa et hsiere Trin staaer denne Digtart i nogle Psalmer , Ps . 1. 32. 37. 49. 50. 73. 132 ; og navnlig i lobs Bog og Prcedikerens Bog . Den hebraifle Poesies Forfljel fra Prosaen er ikke saa meget et Versemaal , men snarere en <b>vis</b> Anordning af Ordene i Henseende til Stavelseantallet, og af Tankerne med Hensyn til deres indbyrdes Forhold , om de ere hverandre sideordnede eller underordnede eller modsatte eller fortscettende . Pontius ( udledes af Bro ) see Pilatus . Poutus var et Landskab i Lilleasien , der nccvnes i Forbindelse med Cappadocien og hvori der boede Isder , Ap . Gj . 2 , 9. 1 Pet . 1 , 1. Mithndaterne havde oprettet et eget Kongcrige med dette Navn , der ogsaa under de romerske Keisere brugtes om hele ' den Kyst-

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014110428001'target=_>, 1851, Menneskets sande Frihed</a><h4>1990</h4>Indromme maae vi forst og fremst Drankerens mcegtige Lidenskab . Vi vide kun altfor vel , at deres Tal ikke er lidet , som ikke for nogen Priis vilde savne Brcendeviin, som , om det ffulde komme an derpaa , vilde sige : „ Tag fra mig Penge og Gods , Huns og Gaard , tag fra mig Styrke og Helbred , tag <b>Kone</b> og Born , tag Kirke og Sacramente , men lad mig tun beholde mit Brcem de v i i n . " Saa overmande vanskeligt imidlertid Foretagens det end kan vcrre , saa folger dog deraf ingenlunde Umuligt heden af dets heldige Fremgang . Hos Gud er ingen Ting umuligt , mindst det , som sigter til Menneskenes Frelse og til hans Navns ? Ere . Dette bevise allerede de glcedeligste og meest opmuntrende Erfaringer . Hvad der i Amerika , England , Scotland , Irland , Finnland , Nusland , Sachsen og mange Egne i vort prenssiffe Fædreneland har havt den bedste Fremgang , det maa overalt vcere udforligt , det vil heller ikke hos os blive uden Fremgang . Forovrigt maae vi med Hensyn paa ovenncronte Indvending heller ikke lade übemcrrket , at Maadeholdsforeningernes <b>Formaal</b> ikte alene bestaaer deri , at komme afgjorte Drankere til Hjcelp og at redde dem fra Undergang , men ogsaa i Særdeleshed deri , at bevare Andre derfra , at danne en Mei nighed , i hvilken Brcendeviinsdjcevelen aldeles ikke har nogen Deel , som allsaa kan vcere ganste sittret for denne Fare . Umaadelighed er vistnok mulig i enhver Nydelse , men forinden en saadan truer Samfundet med saa stor og almindelig Fare , som i Nutiden Brcendeviinsnydelsen , have vi heller ikke Aarsag til at modarbeide den ved sceregne Foreninger . Vi bekjcempe de FieNder , der allerede haardt trytte os , men de , som forst skulle komme eller muligen kunne komme maae vi oppebie . Andre see med en <b>vis</b> Foragt ned paa et Foretagende, der i sin Begyndelse er lidet og uanseeligt . „ Hvad kan en Forening , " sige de , „ der indskrænker sig til et Par usle Landsbyer , gavne Staten ? Sagen maa , naar den stal faae nogen Betydenhed , drives i det Store , og derfor sirax fra forst af gjore en stor Begyndelse . ' Maadeholdsforeningen maa erholde en Indretning , efter hvilken den strax fra Begyndelsen af i det Mindste omfatter en heel Provinds . " Paa dette stolte Ord svarer Herren selv : "Himmerige ligner et Senepskorn , hvilket er vel mindre end al anden Scrd ; men naar det voxer op , er det storre end

<h4>2477</h4>i Herlighed " osv. ( 1 Cor . 15 , 42 — 44. ) , og jeg viste hende nogle andre Steder , hvor samme <b>Lcerdom</b> udvikles . Hvo som fra sin Barndom er bleven fortrolig med Tanken om en Legemets Opstandelse til Liv og Udødelighed , kan neppe forestille sig , med hvilken Gloede den dsende Lovise blev opfyldt , da der for fsrste Gang opgik for hende et Lys over denne store Sandhed . Hun saae nu , at hendes Freb sers store Naadsiutning gaaer ikke alene ud paa at fornye , hellige og forklare kun een Deel af vor Natur , men paa at belive og fornye vort hele Vcesen til Legeme , SM og <b>Aand</b> . Hun sagde : „ Det er nu saaledes for mig , som om alene dette hidindtil havde manglet , for rolig at kunne doe . Jeg fandt det svcert at stilles fra Legemet for evigt ; det syntes mig , som om det ikke kunde blive fuldkommen vel for mig foruden det . Men nu er jeg hjulpen ! O , hvilken Naade er det , at ogsaa mit fattige , brsstfcrldige Legeme stal forts ? ses fra Dsdens og Djievelens Vold ! Ved disse Ord var hendes Ansigt ligesom oversiraalet af Gloede , og hun syntes at vcere fcrrdig med gamle Simeon at udraabe : „ Herre , nu lader Du din <b>Tjenerinde</b> fare , i Fred , thi mine Dine have seet din Frelse . " . Lovises villige <b>Aand</b> behsvede ikke at vente lcenge paa Afsteden fra denne Hytte . Jeg fik Underretning om , at hun var bleven sygere og sengeliggende . Uden Forhaling gik jeg til hende . Hun syntes ikke at virre sit Endeligt saa ncer , men da jeg traadte ind , sagde hun : „ O hvor jeg har lcengets efter at De skulde komme ; jeg gaaer med sicerke Skridt , forlad mig ikke ; jeg snster at dse i Eders Arme . " Jeg satte mig , og lovede at blive hos hende . Men hun vilde ligesom fceste mig ved sin Side og sagde : „ Lwg Eders .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100603017'target=_>Pontoppidan, Erik, 1850, Collegium pastorale practicum</a><h4>364</h4>og farligt Evangelium , en trsstelig <b>Lcerdom</b> for gammel Adam . Dette sit eget kan han ikke bare sig for at blande i Guds , og derved dcrmpe dets Kraft . Ia mangen Sandhed tsr han aldrig sige ; mangen Materie tsr han ei rsre ved , fordi baade hans egen Samvittighed og hver Mands <b>Kundskab</b> <b>moder</b> ham , og stopper Munden paa ham . Saa behsves det da saare hsiligen , at en Christi Tjener , som har det Hoved-ATrinde , at fore andre Ejcele til Christum , selv fsrst maa voere en Christen , salvet med Christi <b>Aand</b> , voebnet med Christi Sind , og gjennemtrcrngt af Christi Kjcrrlighed ; thi ellers kommer hans herskende Egenkjcerlighed , Kjsdets Sands og Verdens <b>Aand</b> til at blande sig i alle hans Embeds-Gjerninger , scrrdeles i hans Pr<?-dikener, og fordoerve dem ; thi han ssger sig selv , og ikke det Christi er .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012080908032'target=_>Mau, E., 1879, Dansk Ordsprogs-Skat, eller Ordsprog, Skjæmtesprog, Rimsprog, Mundheld, Talemaader, Tankesprog, samt et lille Udvalg af Bibelsteder</a><h4>2712</h4>Et godt Raad skal man lyde , seiv om dit kommer fra en Fju.de . — Blandt mange Venner brag faa til hemmelige Ra ad ! 2. 210. — Tit raa di- <b>Kvinder</b> vel . men tiere Mændene . 2,210 . — Et ondt Ha ad kan en <b>Kvinde</b> saa snart tinde paa . som en Mand . 1. 343. - Naar intet Raad vil hjælpe , brager man tit Uraad ( o : Trolddom og ulovlige Midler ) . 2,211 . - , Godtßaad kjendes bedst at ' Mange . 1. 338. ( véd den Ene det ikke . saa vel den Anden ) . — Største Hol , øver Raad . 1. 338. — Ilt godt Kaad kan man vel give . men ikke en god ( ( pførsel . Gode Kaad hjælper Intet mod Ulykken , nåar de ikke følges . 1. 342. - ■ Et godt Raad er ofte bedre end mange listig * Anslag . 1. 342. - - Godt Raad . vel betænkt Mod er i alle Sager god . 1. 45. ( Kaadgiveren skal tørst overveje Alt vel . hvad der kan modi- og folge paa » . — Gode Kaads Foragt er Begyndelse til Ulykke . 1. 339. - Ondt Raad er Begyndelse til Eald . 1. 343. — Eiter Gjerningen sér man. hvori Raadet hai ' fejlet , 1. 341. - <b>Vise</b> Raad bringer ofte stor Nytte tor . • nen der ligger store Ulykker bag . 1. 339. - De værste Kaad blive tit de bedste . 1. 342. — Hvo som giver eller følger et godt Kaad . er viis ; hvo hverkén giver eller følger , er egensmdig. 1. 338. — Unge , Fremmede ogEgennyttige raade intet ttodt - ' • MQ - Andægtig Bøn og <b>vise</b> Kaad forhindre Meget . 1. 340. — Tag Gud i ( paa ) Raad med. saa bliver bans Bistand hos din Forstand baade i Kaad og Daad . 1. 339. — Raadslaa om det Nærværende , befal Gud det Tilkommende ! 1. 339 - Har dn hundrede Raadgivere , saa forlad dog ej dit • Kaad ! 1. 338. Tag tverken en blind Leder eller en daarlig Raadgiver . ( „ thi daarlige Anslag ville ej lykkes altid " ) . 1 - 343. — Lier ej af den Uherde , og tag ej Raad af en Daarel Moth . — Det . som forkynde- ved Raad , afvendes ej ved Raa d . 1. 4 : > ( > . ( om Tegn og Meerker ) . Herre kjære ! gjør Akitofels Kaad til Daarlighed ! 2 Sam . 15. 31. ( Y . 34 : . . til tntet " ; 17. 14 : Es . 8. 10 ) . — Hos Gud er <b>Visdom</b> og Vælde ; Ham tilhører Kaad og Forstand . Job- B . 1 - J 13. ( jvf . Ordsp . 8. 14 ) . - - Salig den Mand , som ikke vandrer i de Ugudeliges Kaad . Ps . 1 , 1. - Herrens Kaad bestaar evimlelio . p B < m _ 1 L ( j yf > or ( lsip 2 , 3 ( ) Qg 2 ] J _ _ Af Mangel paa Kaad falder et Folk : hvor der er mange Raadgi vere . dér er Redning . Ordsp . 11 , 14. - Adlyd Raad og tag imod Tugt , for at du til sidst kan blive viis . Ordsp . 19. 20. 1 | v ! - Sir - 6 > 25 ) - Kaad i en Mands Hjærte er dybt Vand , men en forstandig Mand drager det op. Ordsp . 20 , 5. — Herren er underlig i Kaad og stor i <b>Visdom</b> . Fs . 28 , 29. ( jvf . • ler . 32 , 19 ) . - Begjær Kaad af Hver . som er forstandig , og

<h4>3191</h4>. leg [ Herren ] vil hjemsøge deres Overtrædelse med Riset . Ps . 89 , 33. — Hvo som sparer sit His . hader sin Søn . Ordsp 13 , 24. - Daarlighed er knyttet til den Unges Hjærte ; Tugtens Ris skal drive den langt fra ham . Ordsp.22 , 15. — Vægre dig ikke ved at tngte den Unge ! thi slaar du ham med Riset dø ' r han ikke deraf . Du skal slaa ham med Riset og frelse hans Sjæl fra Helvede . Ordsp . 23 , 13. 14. - Ln Svøbe er til Hesten , en Tømm , til Asenet ( = Æslet ) o , et li is til Daarens Ryg . Ordsp . 26 , 3. - Ris og Revselse giver <b>Visdom</b> ; men en Dreng , som overlades til ( ell . raader ) sig seiv . gjør sin <b>Moder</b> Skam Ordsp . 29 , 15. _ Hvad ville I ? skal jeg [ Paulns ] komme til Eder med Ris eller med Kjærlighed og Sagtmoditfieds <b>Aand</b> ? 1 Kor . 4 , 21.

<h4>10045</h4>sørge for det Forbigangne eller glæde sig for det Nærv»rendes Skyld . 1. 483. — En Viis skal taale de Ondes Laster uden Vrede , deres Velstand uden Avind . 1 , 483. — <b>Kvinder</b> . Vin , Penge og Ære bedrage de <b>Vise</b> . 1 , 483. — En viis Mand har ofte bragt en Xar et Glas Vin og seiv gjort Beskéden ( Besktd ) . 1 , 484. — Det er bedre at have at gjøre med en Nar , end ( med ) en halv-viis ; thi denne vil altid være den klogeste . 2. 229. — Megen <b>Visdom</b> , megen Besværing ( Besvær ) . 1 , 259. — <b>Visdom</b> retter forrige Misbrug. beskikker nærværenue Ting og foresér iforudsér ) de tilkommende. 2 , 289.

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011051004069'target=_>Ewald, H.F., 1875, skotske Kvinde paa Tjele</a><h4>1716</h4>indviet og breven <b>Kvinde</b> , og hans snu Sjcel ydede hendes Dygtighed oprigtig Anerkjendelse . Det Studmm , han drev med gamle Mari , var ham nu vistnok til stor Trdkort og dcemonisk Glcrde , men dog tun et Skalkeskjul : thl det , som han fornemmelig Me hos Troldkvinden , var Kjendstab til de Voer Munkes og Vissing Nonners Synderegister sor ved dette sit Spionen at sin Magt . Nogle rundelige Gåver gjorde den gamle Her . temmeUg aabenmundet , og der undslav hende Tilstaaelser , hvorved hun blottede sig selv og kom i Vigands Magt . Hun blev da et ftieligt Redskab i hans Hcender , og ester hans Begjcering sledskede hun for Margaret og strcebte at indsnige sig i hendes Gunst . Dette lykkedes hende for en Deel ved den ivrige Andagt , hun bar tU Skue , men dog endnu mere ved sin Dygtighed : Lcegekonsten og sit Kjendstab til de medicinfke Urters Egenstaber : thi Margaret strcebte med stor Iver at erhverve sig en Indsigt , som dengang , da der ncesten mgen Lceger fandtes , i en barmhjertig <b>Kvindes</b> Hcender maatte blive til stor og Lindring for de fattige og hlcelve- Syge , men som iMrigt ikte ganste fattedes nogen ! huuslige Dyder oplcert Jomfru . Det var altsaa egentkg den fcelleds Kjcerlighed til Konsten , der knyttede den rene , HMndede UngsOer og den gamle , Hex Ul hinanden . Dog var det saare langt fra , at gamle Marr skulde have vundet noget HerredMme over Margaret : tvertimod maatte hun lMe Bebreidelser for sin Fcerd ssm Besvcergersie og Spaakvinde : men Margaret havde fattet en <b>vis</b> Godhed fyr hende , ynkede hende i hendes Alderdoms SWbelighed og gjorde sig den unywge Ulerkghed at bekjcemve hendes aandelige Vildfarelser . Saaledes var det Forhold beskaffen , som Folmer Rud nys havde omtalt med forklarlig Forundring og Harme . „ . , . , . Vigand havde ofte tcrnkt ved vaakommende LeMgyeo at drage sig dette Forhold til Nytte , og vaa det var den skumle Plan bygget , som nu ftrte ham til Mans Hytte . Doa ilede han ikke til <b>Maalet</b> , men ncermede sig det tFvende; thi uagtet den Tanke , at bortftre Margaret , alle-

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081903009'target=_>Nissen, Johan., 1865, Nissens Samtaler over Luthers lille Katechismus</a><h4>284</h4>forglemme 1 Kor . 4 , 7 : hvo giver dig Fortrin ? og hvad haver du , som du ikke haver annammet ? men dersom du og haver annammet det , hvi roser du dig , som om du ikke havde annammet det ? Den altsaa . som giver sig selv VEren for hvad han formaar at udfsre , han forguder sig selv . Se derimot » paa Peter i Ap . Gj . 10 , 26 og Paulus i Ap . Gj . 14 , 15 ; de give Gud alene 3 Eren . Jer . 9 , 23. 24 : en <b>Vis</b> rose sig ikke nf sin <b>Visdom</b> , og en Stelt rose sig ikke af sin Styrke , en Rig rose sig ikke af sin Rigdom ; men hvo . som vil rose sig , rose sig af dette , at han forstaar og kjender mig , at jeg er Herren . 1 Kor . 15 , 10 : af Guds Naade er jeg det " jeg er . Og det erkjender Paulus , omendstjsndt han i samme Vers erklerer , nt hnn har arbeidet mere end de andre Apostle ; han giver dog Gud Wren . Og merker Eder det Bsrn , at Intet sna snart styrter Mennesket , som nåar han i sit Hovmod og Stolthed setter sig i Guds Sted . Adam vilde « vere som Gud . " I kjeude uok det Ordsprog : Hovmod staar for Fald . Exempler derpaa af Bibelhistorien : Goliath . Senacherib . ( Es . 37 , 25. ) Andre Mennesker gjsr man til Afgud , nåar man frygter og elsker og forlader sig mere paa dem , end paa Gud . Af Frygt for Mennesker lod f . Er . Herodes lohaunes den Dsber halshugge ( Mth . 14 , 9 ) , og Pilatus dsmte Frelseren ( Joh . 19 , 12 ) . Ypperstepresten Eli straffede ikke sine ugudelige Ssnner , fordi hnn elskede dem mere end Gud ( 1 Sam . 2 ) ; og mange Foreldre have handlet som ham . Og hvo , som elsker Fader eller <b>Moder</b> mere end mig . er mig ikke verd , og hvo , - som eister Ssn eller Datter mere end mig . er mig ikte verd . Mth . 10. 37. Mange ere de. som have forladt sig mere paa megtige Krigshere og rige Venner end paa Gud . Men lenmins siger 17 , 5 : forbandet er deu Maud , forn forlader sig paa Mennesket og holder Kjsd for sin Arm , og hvis Hjerte viger fm Herren . Hvorfor stal en saadan Mand forbandes? Han setter Mennesket i Guds Sted . Man maa adlyde Gud mere eud Mennesker Ap . Gj . 5 , 29. Andre Ting . Det er hovedsagelig to Ting , som Mennesket gjsr til sin Afgud : verdslige Nydelser og Penge . Verdslige Nydelser . I Phil . 3 , 19 hedder det om Nogle , at Bugen er deres Gud . Saadanne kjende intet Hsiere , tenke paa Intet , tale om intet andet end Mad og Dritte . Saaledes var det med den rige Mand . som hver Dag levede i Herlighed og Glede . 1 Joh . 2 , 15 siger : elsker ikke Verden , ikke heller de Ting , som ere i Verdeu ! Thi " Alt det , som er i Verden , Kjsdets Lyst og Omenes Lnst og at hoffeldigt Levnet , er ikke af Faderen , men af Verden . Andre snste at hsre , hvad der er tiltrekkende og behageligt: Skuespil , Musik , Dauds , Selflab o . s . v. Og med et saadant paa det nut Verdslige rettet Sind staa de op og gaa tilsengs . Verden og dens Gleder er deres Gud . Og det er desverre Tilfeldet med Millioner af Mennester . Hvorledes er det med os ?

<h4>314</h4>i . Hvad det er at forlade sig paa Gud . Ps . 118 , 8 : Det er bedre at tro paa Herren end at forlade sig paa Mennester . Den , som forlader sig paa Gud og eve og alene bygger paa ham , han kav som Ps . 37 , 5 siger velte sin Vei paa Herren ; han kan haabe paa ham , <b>vis</b> paa , at Gud stal gjsre Alt vel , Alt til det Bedste . At forlade sig paa Gud er altsaa tryggeligen at stole paa ham i alle Ting og i alle Tider og ikke at vente Hjelp af nogen Anden end Gud . Hvad er altsaa det at forlade sig paa Gud og hvorledes viser jeg , at jeg forlader mig paa ham ? 2. Vi skulle alene forlade os paa Gud . Saa forlanger det fsrste Bud . De , som ikke tro Gud , ikke have Tillid til ham , de have Mistro og Mistillit ) til ham . Men Skriften beder os Ebr . 10 , 35 , at vi ikke stulle kaste vor frimodige Tillid bort . Hjobs <b>Hustru</b> kastede sin Tillid bort , da de oude Dage kom , og sagde til Hjob : holder du eudnu fast ved din Gudsfrygt , velsign Gud og ds. Mennesket kan ikke vere uden Noget at forlade sig paa og stolte sig til ; og det udvelger dertil mange Tiug her paa Jorden , som Skriften advarer imod . 1. Egen Klogskab og <b>Visdom</b> . Mange sette sin egen Forstand i Guds Sted , gjsre sin egen Indsigt og Forstand til Gud og mene uden Gud at kuune gjsre sin Gjerning og sit Arbeide . Saadauue glemme leremias ' Ord 10 , 23 : jeg ved Herre ! at Menneskets Vei staar ikke til ham , ja . det staar ikke til ham , som vandrer , at han kan befeste sin Gang . Menneskets Hjerte optenker vel sin Vei , — men det gaar kun daarligt med hnns optenkte Vei . hvis ikke Herren stadfester hans Gang . David bad . at Gud skulde gjsre Achitophels Raad til . Intet , og hau gjorde det . Hvad lere vi deraf ? 2. Rigdom . Ordspr . 11 , 28 : hvo , som forlader sig paa sin Rigdom , han stal 4

<h4>1163</h4>for sit Bryst . En hovmodig Beder have vi derimot ) i Phariseeren : leg takker dig Herre o . f . v. Hvorledes kunne vi da lere at bede med Ndmyghed ? Alene ved den <b>Bevidsthed</b> : jeg kan Intet fordre af Gud ; det er altsammcu , hvad han giver , ufortjent Naade . Henstil det derfor i alle legemlige Ting som Sygdom , Fattigdom o . s . v. ganske til Gud , om hun vil hjelpe , hvorledes ban vil gjore det og nåar han vil gjsre det ; hengiv dig til ham i Bsn og ver saa stille for ham , om han tover med at hjelpe ; ver <b>vis</b> paa , at ogsaa det kommer fra ham . Tag saa ogsaa Lidelserne fm Guds Haaud med Tak , og brug dem til din Omvendelse . Tag fra mig Herre , hvad der drager mig bort fra dig . Tag mig . Herre , og gjsr mig ganske til din . 3. Troende . Inc . 1 , 5 — 7 : Den , som mangler <b>Visdom</b> , ban bede Gud , saa stal den gives ham . Men Betingelsen er at bede med Tro . Det Modsatte deraf er at tvivle , men om en Tvinlende siger Jacob , at han Intet stal faa . Hvad en troende Bsn er , forklarer Frelseren os selv i Mn . 11,24 : Alt . hvad I begjere , nåar I bede . tror , at I stulle faa det , saa stal det vederfares Eder . Min Bsn er altsaa troende , nanr jeg har den faste Forsikring , at Gud bsnhsrer miv Bsn . Denne Tillid og denne Forvisning udtale vi i vor Bsn ved Ordet Amen , som betyder sandelig , visselig. En troende Bsn viser sig ogsaa derved , at man el vedholdende i Bsnnen , om Vsnhsnlseu end ikke kommer strax . Rom . 12 , 12 : verer varagtige i Bsnnen . Ogsaa i denne Henseende maa vi nevne Abraham som Exempel , der sex Gange bad for Sodoma . Jesus bad tre Gange i Gethsemcme , og Paulus bad Herren tre Gange om at tåge sta ham hans Torn i Kjsdet . Den kananeiske <b>Kvinde</b> ( Mth . 15 ) staar i denne Henseende som et herligt Exempel . Herren raaber ogsaa til hende fuld af Forundriug : o , <b>Kvinde</b> , din Tro er stor ! Herren har i Lue . 18 fortalt os en Lignelse om Vittningerne af eu vedholdende Bsn : Lignelsen om den uretferdige Dommer . 4. Frimodig . 1 Joh . 5 , 14 : Dette er den frimodige Fortrsstning, som vi have til ham , at dersom vi bede om Noget efter bans Villie . hsrer han os . Frimodighed i Bsnnen kommer af Troen paa . at Gud hsnr en Bsn . Vi skulle trygle og bede , siger Luther , som de kjen Bsrn bede deres kjere Fader . Bsrnene bede bestandig med Frimodighed i deu gode Tro , at de skulle blive bsnbsrte. Hvorledes vor Von stal vere beskaffen , det har Luther ved sit Exempel vist os . En af hans Venner forteller : Ingen Dag gaar hen . uden at Luther idetmindste tager tre Timer til Bsn . leg len engang havi den Lykke at hsre ham bede . Hjelp Gud ! Hvilken <b>Aand</b> , ' hvilken Tro der er i hans Ord ! Han beder saa andegtig som En , del taler med Gud ; med et saadant Haab . med en saadan Tro som Eu , der taler med sin Fnder . leg ved . sagde lian , at du er vor kjere Fader , derfor er jeg <b>vis</b> paa , at du vil

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012070608274'target=_>Bernhard, Carl, 1878, Krøniker fra Kong Kristian den andens Tid</a><h4>1689</h4>Kjcerlighed , og Gundel maaste efter at hore denne Tilstaaelse , vilde af dem have undvceiet den store Nydelse , at udtrykke deres Fslelser i det daglige Livs Talemaader og omdanne de almindelige Scetninger til Hjartets glsdende Sprog , forstaaeligt og Yndigt for de indviede , og tomt og indholdslsst for ethvert fremmed Vre . HosKlavs Daa var denne Kjcerlighed ovstaaet pludselig , ncesten ved det fsrste Blik , som det faa ofte sier hos Mcend , der ere begavede med et varmt Hjcerte og en levende Indbildningskraft. Gjennem Djet havde hendes Elflvcerdighed fnndet Vejen til hans Hjcerte , og dette var maaste Me siet , hvis Gundel ikte havde vceret faa smuk , at han var forelstet allerede den fsrste Gang han faa ' hende . Hendes Kjcerlighed derimod var vozet langsomt , nceret af langt aandeligere Fode , fom det i Almindelighed er Tilfceldet med <b>Kvindens</b> Kjcerlighed , der ikte engang i sin Fremspiren er saaledes knyttet til det ydre , som Mandens . Hun er fra den tidligste Barndom vant til at <b>vise</b> sine Tanter og Folelser tilbage i sin egen Barm , de faa en gansie anden Betydning i hendes stille , enssormige Liv , og nåar de omsider henvendes paa den Gjenstand , om hvem de saa ofte og faa ssdt have dromt , da voxer Kjcerligheden langsomt og klarer sig selv til <b>Bevidsthed</b> , idet den atter gjennemgaar de ofte dromte Stadier , Skridt for skridt , indtil den omsider nåar en Styrke og en Renhed , som maaste kun <b>Kvindens</b> Kjcerlighed er i Stand til at opnaa , - og som Mandens Kjcerlighed itte altid er i Stand til at paastjsnne . Dette var dog ingenlunde Tilfceldet med Klavs Daa , thi han baade fattede , paastjonnede og gjengjceldte Gundels Kjcerlighed . Og i deres stille Tanter dromte de sig begge den lytteligste Fremtid , idet alle de udvortes Betingelser for en lykkelig Forbindelse vare ens . I Fru Anne Arvidsdatters aabenbare Godhed for den unge Mand troede de at se en stiltiende Billigelse af den under Modersjet fremspirende og voxende Kjcerlighed , og en Erkjendelse af , at det faderlige Samtykke ikke vilde blive ncegtet . At Fru Annes <b>Lcerdom</b> hverken tillod hende at hore eller se , faldt ingen af dem ind med en Tanke . —

<h4>2443</h4>Fra det skotske Herberg gil han til Landslncegtenes Kvarter for at <b>vise</b> dem den samme Hoftigheo og se til sine Venner blandt Tysierne . Han fortalte dem , at Stotterne allereve holdt sig lystige uden for Herberget i Stindergaden , som om det var Midsommers , og at de havde spottet Tyslerne i deres kaudervelske Sprog , der lod som Hundegjsen og ilte som et mennesieligt Tungemaal . De sagde om dem , at de havde nok en torstig Lever , men svage Hoveder , nu laa de hjemme paa Bcentehalmen og spyede , thi en tyst Landstncegt tunde ille taale saa meget som en siotst <b>Kvinde</b> . Tysterne bleve varme i Hovedet over denne Tale , og de begyndtc at svange og bande med hidsige Eder , De svore , at de vilde tilmtetgjore alle de Skorter , der vare i Veroen , og nåar det var gjort , vilde de drage til og taste dem , der vare hjemme , i Stranden . Tbi det var en at der skulde hores andet <b>Maal</b> over hele Jordens Kreds end tyst , som var det Sprog , Gud Herren havde talt , da han havde stabt Verden , og som vore fsrste Forceldre havde talt i Paradiset , hvad hver Mand vidste . Men uagtet de vare meget vrede , vilde det nceppe vcere lykkedes Broder Martin at drive dem til at forlade deres Kvarter for at trceve Stotterne til Regnstab for deres dristige Tale . Thi dels var Mandstugten strceng iblandt dem , og dels vidste de as Erfaring, at Stotterne sloges fom fortvivlede , nåar man kom dem for ncer , og havde en ualmindelig Fcerdighed i at bruge deres lorte Svcerd i Haandgemceng , hvortil Tysiernes lange Degener ikte vare saa stikkede .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008091002003'target=_>, 1849, Vor Herres og Frelsers Jesu Christi nye Testamente, med Davids Psalmer</a><h4>569</h4>dc Skriftkloge . « Mnttd.7 , 2 B.sq . > 83. 23. Og 6 der var et Men » - forså Nesse i deres Synagoge mcd , 34. en uren Anlid . og hnn rnnbiejsom hsit . " Lue . 4,33 ! Syg 24. Og sagde : cia ! « hvad Dia ' hnve vi med dig nt skaffe . lEsn tnle - Nnzarene ! er du tommen nt " Ap forrcrrvc os ? jeg kjender dig . 35. hvo du er . ten Guds Hellige , det , " r - 5 7. Matth . 829 han 25. Og lEftis truede ham , til " on snare : ti , o ^ far ud nfhnm ! 26. Og te » urene <b>Aand</b> siet 36. hnm . og ranbtc med stor Rsst , z vare og foer ud nf ham . j ter l 27. Og te blcvc alle forfar- 37. tete . lan at de bespurgte sig i sagd med hverandre , og sande : hvad efter er det ? hvad er denne for " en 38. ny <b>Lcerdom</b> ? thi han byder og ! ladci de urene Aandcr mcd Maqt ^ smac og de lyde hnm . Ap . Gl7 19. l 28. Men hans Nygte udkom derti sirar i alt det omkringliggende . Land i Galilan . > 39. 29. Og cc git strnr nf Sy- ! res nngoqcn , og kom i ' Simons lcen ; og Andrens Huus . med Ja- 40. kobus og Johannes . til b ' " Lue . 4,38 . paa 30. Men Simons Hustrues til h <b>Moder</b> laae . og havde Feber ; du r on strår sagde de ham om bende , j 31. Og han gik til hende , i 41. ioq fat paa hendes Hnand , og j derli reiste hende ov og Feberen ! og , forled hende strår ; og hun ! til b

<h4>949</h4>vis Grunt pnn ten <b>Lardom</b> , i hvilke » du er bleven mundt » listen untcrviist . 5. I " Herodis , ludcrce Konges. Dnge . vnr en Prast af > Abire Skifte , ved NnvnZncharias; ost hnns <b>Hustru</b> var af Arous Dottre . og hendes Navn var Elisabeth .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011061608052'target=_>Wexels, W.A., 1846, Forklaring over Det nye Testamentes Skrifter</a><h4>1068</h4>dant Mere , ved " Drukkenstab og Fraadseri " til alslags Umaadelighed, o . s . v. — " De , som gjore Saadant d . e . som saaledes tjene Kjodet , stulle Me arve Guds Rige ; " sml . 1 Cor . 6,9 . 10. Eph . 5,5 . De staae naturligviis , som übodfcerdige , Synden elstende Syndere , aldeles udenfor Naaderiget og kunne saaledes Me heller arve Herligheds-Riget , Me dele Guds Boms Lod ; deres Arv er meget mere Dod og Fordcervelse ( 6,8 . Rom . 8,13 . Aab.21 , 8. ; 22,15 . ) , og denne maae da end vissere komme over dem , der " pacckalde Christi Navn og Me ville afstaae fra Uretfcerdighed , " der fra Aandens vende tilbage til Kjodets Tjeneste ( Matth . 7,23 . Ebr . 2,2 .3 . 2 Pet . 2,22 - 24 ) . Denne Tilbagevenden er jo det Samme som Affald fra Naaden, hvis retfcerdiggjorendeVirkning st and ser , nåar dens helliggjorende Virkning modstaaes , ved hvilken vi stulle blive mere og mere Mede til at see Gud , leve i hans Samfund , og stadfcestes i vort Kald og Udvcelgelse . Sml . 2 Pet . 1,10 .11 . — " St . Paulus lcerer her det Samme , som Christus har lcert i Matth . 7,16 .17 . , nemlig at Gjerningerne eller Frugterne noksom <b>vise</b> , om Trceerne ere onde eller gode , d . e . om Mennestene folge Kjodet eller Aanden ; som vilde han sige : Paa det at Me Nogle iblandt Eder stulle forevende , at de Me forstaae , hvad jeg nu har talt om Kjodets og Aandens Strid , saa vil jeg stille Eder for Oie , forst Kjodets Gjerninger, af hvilke selv de Ugudelige erkjende en Deel , derncest Aandens Frugter . Der vare nemlig Hyklere blandt Galaterne , ligesom paa ncervcerende Tid , hvilke foregcwe , at de vare fromme og gode Christne , roste sig meget af " <b>Aand</b> " og kunde mcmgfoldigen og herligen snakke om Troens <b>Lcerdom</b> , men vandrede Me des mindre ei i Aanden , men i Kjodet , og fuldkommede dettes Lyster ; derfor lader Paulus dem her offentligen vide , at de Me vare de Folk , som de udgave sig for " ( Luther ) . V . 22. I Opregnelsen af Aandens Frugter eller det gjenfodte indre Menneskes Vcesen og Yttringer ncevner han iscerdeleshed dem , der danne en Modscetning til hiin Bitterhed og Uenighed , som begyndte at faae Indgang i Menigheden , og som derfor besynderlige » burde lcrgges paa Hjerte . Forst scetter han Kjcerligheden , som <b>Moder</b> til alle de ovrige , ja til

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011072920001'target=_>About, Edmond, 1880, brav Mands Roman</a><h4>136</h4>han fortjente at elstesl <b>Kvinderne</b> er lloge Dommen , for dem lan man hverlen stjule de mindste Handlinger eller de hemmeligste Tanter ... " Saaledes vuggede min <b>Moder</b> mig til langt paa Dag , idet hun overoste mig med fit Overmaal af Smerte og Pmhed , uden a ! hcevs sin S ! emme og uden alle heftige Udtryl eller dramcttiste Bevcegelser . Hun forlalte mig om sit lykkelige Husliv , forn om jeg itte felv havde vcere ! Vidne dertil ; hun priste Taalmodigheden , Blidheden , Finheden og den fonlommende Opmcrrlsomhed hos denne Mand , forn hun nu havde tabt . Alt dette blev fagt saa mildt , og jeg lyttede til hendes ljcere Stemme med en faadan FMlse af Ro i min Sorg , a ! jeg indimellem maatte spMge mig selv , om vi virlelig led nol nu , eller om vi itte igrunden var Afstum , og vore Taarer unaturlige . Et enefte Ord fortiande mig denne elstende og elstede <b>Kvindes</b> Resigncttion . „ Og tilsidft , hvad er det saa ? " sagde hun ; „ det var hans Plig ! ! Hust hans sidste Ord her og dernede . Han stulde do " for de andre , og vi stulde blive igjen uden ham . leg ved itte , om de Folk , han har hengivet sig for , nogensinde vil gjengjelde os dette , men hvad gjM de ! ? GjM som han ; gjM hvad der er Din Plig ! ! <b>Vis</b> Dig i Liv og i DM som en SM , der er ham vcerdig ! . . . " Medens vi faaledes stille llagede i den ! omme Sive , begyndte det atter at gaa og lomme omkring os . Vasset havde intet glemt : sten Bud var strax afsendt for at bringe den sMgelige Nyhed ti ! Lcmnay og de ncermeste Landsbyer ; to Drenge arbeidcde i Vcerlftedet lang ! nede i Haven en stor Liglifte af Egetrce ; Slrceddenn , forn havde mit <b>Maal</b> , arbeidede mig for ! e Klceder , og Ccttherine , dm brave <b>Pige</b> , forn var stint til alt. syede en sort Merinos Kjole til min <b>Moder</b> .

<h4>1431</h4>Efterat jeg saaledes havde faaet Vished i min Antagelfe og havde bane ! Veien for Udstrelsm af mine Planer , lod jeg nogle Dage hengaa , uden at fontage noget fcrngent . Medens jeg godt vidfte , at det vilde geraade mit Rygte forn Deteltiv til stor Mn , istid jeg atter lunde overantvorde de to undvegne Forbryden til lustitims Hcender , saa gil paa den anden Side min Mrajerrighed ud ftaa , at indntte det hele saa , at jeg ille blot sit redde Hr. Vlales <b>Hustru</b> fra hendes Vetrcrnqthed og Ned , men at jeg ogsaa sit befri hende fra den offentlige Skandale , forn uundgaaelig vilde vcere Wlgen , nåar ! o utvilsom ! stcrrlt bevcebnede Mcend stulde aabenlyft arresteres . Her gstld ! det altsaa mere at bruge Krigslist end Maat , og foråt naa d.tte <b>Maal</b> , maa jeg basere hele min Taktik ftaa e ! indgaaende Kjendstab ti ! alle mulige Biomstcendigheder , forn derved lunde lomme i Belra « ! ning . Tre Dage anvendte jeg paa at erhverve mig disfe Kundstaber , og Resultatet af mine lagttagelser var stigende Kjendsgjerninger : For det strfte : Min Vcertinde havde Ret i sin Opfatning, at man aldrig lod den unge <b>Pige</b> vcere alene ; thi en af de to , enten Faderen eller Broderen , forblev ftedse hos hende . For det andet : Trods denne strenge Bevogtning lod man h nde dog fri ! gaa op og ned i Korridoren , stjM ! rigtignol lun en lort Stund ad Gangen . For det tndie : Det rMe Kors paa Denn syntes at have en hemmelig Betydning og at ftaa i en <b>vis</b> Forbindelse med Beboernes Fravcenlfe eller Tilstedevcerelfe i de leiede Vcrnlfer ; en Aften nemlig , da alle ! re gil ud ftaa en Gang , blev Korfe ! udvistet , men strax igjen paategnet ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006081800029'target=_>Wexels, W.A., 1860, Vidnesbyrd fra Kirken</a><h4>294</h4>mit naturlige Vcrsen . skrobelig i alle mine Veie , skyldig ester alle Guds Bud , hildet i utallige Lidenskaber , bunden afLcrnker, som ingen Skabnings Haand kan lose , udygtig af mig selv til et Liv i Gud , udygtig til en Handling , om hvilken Sandheds <b>Aand</b> kan vidne : den er reen ; jeg er en Synder , som ikke kan hjcelpe mig ud af min Elende , ja , som ikke seer , hvorledes det er muligt , at jeg kan hjelpes ud deraf ; jeg er et Vsssen , borte fra Gud , og dog med en Gnist af hans <b>Aand</b> i mig , dog uden Hvile , dersom jeg ikke sinder ham , dog draget til ham paa forunderlig Maade , dog ledende , sagende , grublende , sporgende , om jeg kunde finde ham , om jeg kunde komme tilbage til min Fader , dog horende hans Rost , som en almcegtig Skabers , en algod og alvidende Forsorgers , en fon glemt Faders Rost , gjennem tusinde Vidnesbyrd trindt omkring . Hvo jeg er ? Ak , en Skabning , der drages mellem to Verdener, mellem Himmel og Jord , der er hjemme og ikke hjemme i Stovet , der begjerer det Timelige og aldrig tilfredsstilles derved , der higer efter det Evige og tusindgange glemmer det og naaer det ikke ! Hvo jeg er ? ak , en Vandrer i det Fremmede, der soger Hvile , der lamges og venter , og har et saa usigelig dunkelt og vaklende Haab om , nogensinde at komme frem til . Lcrngselens og Ventningens <b>Maal</b> , til Hvilens Land , en Vandrer i Natten , for hvem enkelte Stjerner lyse , men blive saa borte igjen , en Vandrer , hvis hele Vcesen sukker efter Forlosning , efter Frelse , om den er mulig ! — Men , o Venner ! naar vi have lcert at aflcrage dette Vidnesbyrd om os selv , ikke med Forstanden alene , ikke efter et foreskrevet Ord , men af dybeste Hjertens Grund , med den inderlige Folelse , saa er det ! det er jeg ! det maa jeg sige om mig selv ! see , da <b>moder</b> lohannesrosten os , eller rettere , Guds Rest gjennem dette

<h4>637</h4>3 aab , om vor Fodsel og vor Pleie af vor jomftuelige <b>Moder</b> Maria , den christne Kirke , stal Jesu Barndomshistorie og navnlig Dagens Evangelium minde os ) — derfor indpodedes vi som Born ved Daaben i Christo , for at hans Barn cl iv skulde meddele sig til os , at vi , som dobte Bern , skulde ligne vor Broder , Guds og Marias Son , selv Guds Born og Smaa i ham , og saaledes opvoxe til ham , indtil hans Alders fulde <b>Maal</b> ; ( dette giver os et dybt Blik md i Barnedaabens Fornodenhed , Hellighed og Betvdning ; uden den var der jo intet christent Barneliv , i hvilket Herrens eget kunde speile sig , ingen Opvert fra Barnsbeen , fra Vuggen af , til ham , og tcrnk saa , naar Fader og <b>Moder</b> ikke kunde have christne Born , i hvilke den lille Jesus levede ! ) — og derfor males i Chr ist i Barndomsliv et christent Barndomsliv for os , i hvilket Bevidsthcden , den glade Bevidsihed, om at virre et Guds Barn , vaagner og kunstiss afprcrger sig , idet denne <b>Bevidsthed</b> ikke drcrber , men levenoegjor og helliger den barnlige Lydighet » og Glskelighed for Gud og Mennesker , baade i det Hele og i dens enkelte Trcrk .

<h4>1067</h4>hans Helligdom , Ord og Sakramenter , foragtel , forternet og bedrevet , krcenket paa det Allerdybeste Fader og <b>Moder</b> , krcrnket i heieste <b>Maal</b> din Nwste paa Mre , Liv og Gods , ja uden Blussel traadt al Sandhed og Kjirrlighed , al Mrbarhed , Tugtigheb og Ret , under Fod , saa skal alt Tette vcere dig forladt for Christi Pines Skyld , verre udslukket i Forsoningens Naadedyb, som Funken i Havet , og Gud vil ikke see andet hos dig end sin Sens dyre Fortjeneste . — Og du , ChristenM , som bliver lunken i Aarvaagenheden og Bonnen og Kampen , som lader dig friste til , hvad din Samvittighed fordemmer , lader dig friste enten til Lempen efter Verden eller til daarlig <b>aandelig</b> Hovmod , eller til Hykleri for dig selv og Gud og Andre , eller i Modgangs Dage til Mishaab og Utaalmodighed, — Ordet ' , for dig ! for dig ! trcrnger md paa dig , advarende , bedende , formanende , at du stal tcrnke paa din korsfcrstede Kjwrlighed og blues og sige til Satan under enhver Skikkelse , i hvilken han kommer til dig : vig fra mig ! — Og , min Christen ! i den sid ste Kamp , naar Dsdens Skygger indhylle dig , naar Oiet og Hjertet skal briste , naar du stal forlade dette Liv og Alt , hvad Jorden tilherer , naar al Tale om , at " Haabet ikke er forbi , at du maaskee endnu kan leve " , er Daarstabens , Forfcengelighedens og Usandhedens Tale , mens det er det Visse , at nu stal du dcc og kan ligesaa lidt kjeve dit Livs Forlengelse , som du kan gjere en Dag deraf ulevet , da , naar du udstrcrkker det bcrvende , men troende Hjerte mod Guds Naades Favn , da kommer Guds <b>Aand</b> med Ordet : for dig , for dig ! han led og dede for dig ! og ferer det til dit Hjerte , at du skal gribe det , og paa og med det Ord : for mig , for mig ! sedelig indslumre , og gaae retfoerdiggjort md i Guds Hvile , md der , hvor Neveren paa

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008091603026'target=_>Baggesen, Jens, 1847, Jens Baggesens danske Værker</a><h4>4103</h4>Af Charakterer er nu rigtignok ikke een eneste deri , hvis man ellers forlanger af disse , at de i det mindste mciae vcere hele , og svare til dem selv . Dog dermed ville vi ikke regne det saa noie — vi vilde ikke engang berore denne Mangel i et Syngespil , hvis der var noget Andet , og blot noget Syngeligt i samme . Men , Ucharakterer eengang tilglvne , reent tor de dog ikke ophceve stg selv , som de ncervcerende ; thi vil man og i et stettegnet Maleri tåge til Takke med Heste paa tre Veen , kan man dog umulig oversee Koer og Faar paa to - — - iscer naar deriblandt forekomme nogle paa fem eller ser , der , naar man feer sig til , skulle verre de selvsamme — > paa Historiemalerier , f . Gr . , hvor Hjorden kommer gaaende i Vaggrundm og bliver slagtet i Forgrunden . Hvad stal man sige , f . Er . , om <b>Heltinden</b> i denne Hule , der baade er en talende Person , et Ojenfcrrd og en Natur - <b>Aand</b> — > en saa rasende Sammensmeltning, at Recensenten i Athene ( der sikkert er Forfatteren selv , eller hans Muses hjerteligste Tilbeder ) med sin , Himlen veed , dog Intet mindre end Philosophie og <b>Locrdom</b> forraadende Theorie om  menneskelige Aander og Naturgeister " ^ )

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008091003001'target=_>Kennedy, Grace, 1852, Miss Grace Kennedy's samtlige Værker</a><h4>1741</h4>Intet laae Tante Ross mere paa Hjerte , end det , som hun holdt for sine Boms Lykke . Gjerne vilde hun antage den lille foreldrelose Anna som sit eget Barn og anvende endeel af sin Omsorg paa hende . Ja det var hende endog meget kjert , at Antallet paa hendes Bern blev foroget med et til , foråt hun ogsaa paa dette kunde <b>vise</b> Fortreffeligheden af sin Opdragelscsmethode. Tante Ross sparede intet , foråt naae dette <b>Maal</b> . Hendes holeste Vnste var , at hendes Bern stulde blive artige , belevne og af fim <b>Dannelse</b> . Intet er imidlertid vanskeligere end at bestemme , hvad det er at vere artig og beleven . Dette troede Tante Ross , hun forstod fra Grunden af , - medens dog andre fine og dannede <b>Damer</b> loe af Tante Ross ' s Negreber og ansaae hende for en meget almindelig , udannet <b>Kone</b> , og medens disse , som loe af hende , atter bleve udleede af Andre . Tante

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011072920003'target=_>Nathusius, Marie, 1880, Elisabeth</a><h4>232</h4>kjcere Datter i denne Kreds lcerte at kjende en fornem , men paa samme Tid brav Mand , — at hun skulde kunne vcelge en uvcerdig Mand vil man ikte engang tcenke sig Muligheden af . Og hvor skal overhovedet en ung <b>Pige</b> gjFre Bekjendtskab med unge Mcend undtagen i sterre Selfkaber? — Hvis Moderhjertet stammer sig over saadanne Tanker og ikke vil lade dem opkomme , da ncermer Forfl<reren sig paa en anden Maade og siger : ~ En ung <b>Pige</b> kan ikke udvikles og uddannes uden ved Omgang med andre Mennesker , — kun derved kan hun blive fast og selvstcendig. Ensidighed og Hovmod er Feil , som bedst bliver modarbeidede ved at man kommer sammen med andre , iscer fremmede og os overlegne Mennestcr , altsaa kan det under visse Omstcendigheder vcere ret godt at ftre sin Datter ud sra Hjemmet og ind i Verden ; det vilde dog vcere sorgeligt, ' hvis hun ikke stod saa fast i sin Kristendom , at hun var hcevet over de Farer , som der vil hende . " — — Men saadan Verdensklogskab vil hcevne sig . Nei , Moderhjerte, du kan ikke bede for din Datter , hvis du ikke tror , ja , er fast og salig overbevist om , at Herren , som tcrller hvert Håar paa vort Hoved , ogsaa vil ftre den af ham bestemte Mand til din Datter uden dine Foranstaltninger og mennestelige Tanter , og din Datter vil blive dannet , selvstcendig og forstandig ogsaa uden de Midler , som verdslig <b>Visdom</b> raader dig til at benytte dig af . Herren vil gjstre en lldvei , naar du kun vandrer i Troskab mod ham . - — Men med Hjertet opfyldt af saa megen <b>modcrlig</b> Daarskab og Forfcengelighed du ikke bede og anraabe ham om Hjcelp . Tror du ikke , at de Feil , som du i det skjulte ncerer i dit Hjerte , vil straffe sig selv paa dine — la , det er just det forunderlige og hemmelighedsfulde , at om du end ved Ord og <b>Lcerdom</b> ivrigt og flinkt skjuler dine Feil , saa vil de alligevel gaa igjen hos VMnene . Det er Verdensaandens Forbandelse , som ikke lader sig borttage ved smukke Ord . Bclrnene ved ikke , hvorledes det ser ud i Moderens Hjerte , men de dog , at der er noget , som ligger hemmende paa deres egentlige Sjceleliv. Men saaledes er det paa den anden Side ogsaa med Guds <b>Aand</b> , — naar den bor stille og forborgen i Hjer-

<h4>675</h4>talte sine Meninger , undertiden forefaldt Samtaler , hvorunder han havde sat <b>Kvindens</b> Kjcerlighed og Enfold over al <b>Kundskab</b> og alle Gjerninger , og hun havde med stor Smerte maattet tåge dette til sig . men det var kun svage Hentydninger , og hun havde snart overbevist sin forlovede om , at heller ikke hun fatte <b>Kundskab</b> og Gjerninger over alt. Idag havde man i ethvcrt Fald gjort hende Uret . Skulde nu Elisabeth , der as alle blev betragtet som et umodent Barn , givcs hende til Forbillede ? Skulde hun ikke have Lov til at have sin egen Mening ? Var det KjoerlighedslMed , at hun Mskede . Elisabeth maatte vende om til Herren , selv om dette kuv kunde ske gjennem Korset ? Hcndcs Forstand ledie frem den cne Undskyldning mere uholdbar end den anden ; sin Synds cgcntlige Kjerne skiFd hun fra sig og vilde ikke ligeoverfor fin <b>Moder</b> og sin forlovede stige ned fra den Hpidc , hvor hun befandt sig saa vel , og gaa ned i Ydmyghedens Dal ; hun syutes ikte om at blive saa alvorlig mindct om de Ord : ~ Om du i Naade ei ser bort Fra al den Synd , som her er gjort , Hvo kan da frelst vel blive ? " Hun var jo en kristen og det en aarvaagen kristen , der selv vilde vide at loegge sig saadanne Ord paa Sinde . Hun befandt sig nu i en trMeslM Tilstand og var ikke istand til at <b>vise</b> sig ved Middagsbordet . Da Moderen kom for at lMe , hvorledes hun havde det , bad hun om at faa blive uforstyrret , og Moderen trytkede hende under Taarer til sit Hjerte , uden at kunne sige hende noget . Havde hun ikke allercde loenge vceret bekymret for Emilie ? Her i Berlin , hvor hun fandt faa megen Anerkjendelse i en Kreds af troende , var hendes Sikkerhed betydelig tiltaget , og den fremragende , begavede unge Mands Kjcerlighed havde ikke ftrt hendes Hjerte til Ydmyghed , men til Hovmod . At selv hendes forlovede fprgede derover , vilde Emilie aldrig have tilstaaet for sig fclv , men Modercn fMe det med Mngstclse , og en saadan Scene forn den idag havde hun stadig veniet . ~ Herren vil hjcelpe hende , " tcenkte den bedrpvede <b>Moder</b> ; ~ vi maa alle bcere vore Synders Byrde , og Herren alene kan lcere hende at bede om Varmhjertighed . "

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008091503018'target=_>Harboe, Ludvig, 1851, Psalmebog, eller En Samling af gamle og nye Psalmer, til Guds Ære og Hans Menigheds Opbyggelse</a><h4>3488</h4>Epistelen , Gal.4C . IV . Mm jeg siger : saalcenge Arvingen er et Barn , er der ingen Forstjel mellemham og Trcellen , endog han er Herre over alt Godset ; men han er under Formyndere og Huusholdcre , indtil den af Faderen bestemte Tid . Saaledes stode og vi , saa- Iccnge vi vare Bsrn , som Trcelle under Verdens Vorne - <b>Lcerdom</b> . Mm der Tidcns Fylde kom , udsendte Gud sin Son , fodt af en <b>Qvinde</b> , fodt under Loven , Paa det at ban stulde frikjobe dem , som vare under Loven , at vi stulde faae den ssnlige Ndkaarelse . Men eftersom Ida ere Sonner , haver Gud udsendt sin Sons <b>Aand</b> i eders Hjerter , som raaber : Abba , Fader ! Saa at du ikke lcengere er Trcel , men Son ; men er du Son , da er du ogsaa Guds Arving formedelst Christum .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112301025'target=_>Nicolaysen, N., 1875, Norske Stiftelser</a><h4>2751</h4>befulgte og haaber at Vedkommende ville respektere . Erfaringen har nemlig vist , at Gjenemfsrelsen af detaljerede Bestemmelser i lignende Tilfelde oftere er forbunden med Vanfleligheder , naar forandrede Tidsforhold indtrcede , og at saadanne Bestemmelser , om end hensigtsmessige efter de ved deres Fastsettelse raadende Anstuelser , i Tidens Lsb blive mindre passende til de forandrede Omstendigheder . De Bestemmelser, hvortil jeg derfor vil indfircenke mig , ere fylgende : 1 ) Legatets Kapital , der ikke maa anvendes uden til Kjpb eller Opftrelse af Bygninger til Opfostringshuset og til Inventarium til samme , gjpres foryvrigt frugtbringende ved Udlaan mod Pant i faste Ejendomme efter Reglerne for Udlaan af offentlige Stiftelsers Midler , og Renterne anvendes, faavidt fornodiges , til Stiftelsens lovende Udgifter . — 2 ) Stiftelsens Bestyrelse bestaar af 3 Mend og 2 <b>Kvinder</b> ; hine velges saaledes: den ene af og iblandt Fattigkommissionens Medlemmer , den anden af og iblandt Bergens Skoletommissions Medlemmer , og den tredje af begge de nevnte Kommissioner i Forening udenfor deres Midte ; de to førstnevnte Medlemmer deltage i Bestyrelsen , saa lenge de ere Medlemmer af den Kommission , til hvilken de ved Valget hyrte . De <b>kvindelige</b> Medlemmer af Bestyrelsen velges af dennes mandlige Medlemmer. - 3 ) Bestyrelsen har , foruden at fpre stadigt Tilsyn med Stiftelsen , at bestemme , hvilke Bern der stal optages i samme , idet den dog altid har at tåge tilvsrligt Hensyn til Fattigkommissionens og Skolekommissionens Duster i denne Henseende , og ipvrigt at afgive aarlig offentlig Beretning om Stiftelsen . - 4 ) Bestyrelsen stal ansette en ugift <b>Kvinde</b> eller Enke til at vere Stiftelsens Forstanderinde , hvis Lon mindst stal vere 100 Speciedaler aarlig foruden fri Bolig med Lys og Brende samt Kost , og hvis Pligter og pvrige Forhold til Stiftelsen blive at bestemme af Bestyrelsen . — 5 ) De Bsrn , som optages i Stiftelsen , stulle der opdrages , med det <b>Maal</b> for Bje at dannes til kristelige , ordentlige og duelige Tjenestepiger ; Opdragelsen stal derfor vere tarvelig baade hvad Kost og Kleder angaar , men forpvrigt saadan . at Bornene have det sundt og godt ; de stulle saa tidligt som muligt vennes til den almindelige Husgjerning og opleres i de simplere <b>kvindelige</b> Haandarbeider, idet de forsvrigt stulle spge de almindelige offentlige Almuefloler indtil deres , Konfirmation , med mindre Erfaring flulde <b>vise</b> , at dette er forbundet med for store Vanskeligheder , i hvilket Fald der , saafremt Stiftelsens økonomiske Stilling tlllader det , stal oprettes en egen Skole ved Stiftelsen for dens Bsrn . — 6 ) I Stiftelsen maa ikke optages Bsrn eldre end 11 Aar , og naar Bornene opnaa 16 eller hpjst 17 AarS Alder , maa de ikke lengere forblive i Stiftelsen , men Vestyrelsens kvindelige

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011072120001'target=_>Walnum, S.B. Hersleb, 1880, Prædikener over Kirkeaarets Evangelier</a><h4>1183</h4>af Isracls Huus og efter de forvildedc Sjcele blaudt Hedningefolkenc. Dc vilde saa ofte ikke lytte til hans Hyrderß , men flygtede bort fra ham og viste sig maaskee i lange Tider gjenstridige imod ham ; men han bar over med dem i nendekg Langmodighed og vedblev det enc Aar efter det andet at lokke dem til sin Naadcsfavn . Han modtog stcdse med den inderligstc Kjcerlighed enhvcr arm Synder , som endelig lod sig lede til Omvendelse , og overftrte ham fra Merket til Lyset , fra Dpdcn til Livet , fra Satans Magt til Gno . Og saaledes vil han utrcettcligcn vcdblive med at ~ eftersce sine Faar og redde dem fra alle de Steder , hvor de cre adspredte paa en Skycrs og Mnlms Dag " ( Ez , 34 , 12 ) , vil vcdblive dermed , indtil den store Time oprinder , da han kommer synligen igjen til denne Jord , og det stal blive fnldeligcn opfyldt , hvad han har forjcettet : ~ Der skal blive een Hjord , cen Hyrde . " la , i Sandhcd ! lesns er ~ den gode Hyrde " , og ~ Faarenc sknlle hans Vistnok vandr er han Mc lcengcrc blandt os i synlig Skikkelse og taler ikke til os med legcmlig Mund , som han gjorde i sine KjFds men aldrig , saalcenge denne Vcrdcn staar , skal hans HyrderM forstnmmc blandt Adams BMn ; den lyder til os igjcnncm det Sandhcdcns Ord , som han lagde paa Propheterncs , Apostlerncs og Evangclistcrncs Lceber , og som de ved Aandcns Vcilcdning have nedstrcvet ; hans RM lyder til os igjcnncm hans Tjencre , naar de ikke forkyndc deres egen Viisdom , men det rene Gnds Ord i den hellige Skrift , gjcnncm hver Lcerer , naar han nnderviscr de Smaa efter den snnde <b>Lcerdom</b> i Ordet, gjcnncm hver Fader og <b>Moder</b> , naar dc forkynde deres Bprn Korscts hcllige Evangclinm , gjcnncm hver Mcdchristen , naar han lceggcr os et Gnds Ord paa Hjertet . ~ Den gode Hyrdes er en nsvigclig Sandhedens og hvor trcengc vi ikke til at HM den . Om vi end slittigen brnge dc Sjcelsevner , som Gnd ved Skabclsen har givet os , om vi med Opmcerksomhcd betragte Gnds store Gjerninger i Natnrens Rige , om vi med al Flid aatc paa Historiens Vidncsbyrd om Gnds FMlsc mcd Mcnncskcnc , om vi stadigcn lytte til Gnds Stemme i vor Samvittighcd , om vi omhyggeligcn ville gjcnncmgrandste Alt , hvad dc Talcntsuldeste og Knndsiabsrigestc blandt denne Verdcns <b>Vise</b> have lcert o , dersom der " er vaagnet alvorlig Salighcdslcengscl r vor Sjcel , da ville vi finde , at ikke kan vor egen Forstands Lys bortjage Merket , som omgiver vor <b>Aand</b> baade i Livet og i ikke knnnc denne Verdcns <b>Vise</b> med deres dybcstc

<h4>1635</h4>tabes , dersom ikke vor Sjcelehyrde selv os paa vore vildfarmde Veie og hjalv os med utrcettelig Kjcrrlighcd ; thi en saa rcedsom Magt har Sjcelemorderen faaet over os , at vi aldrig af os selv vilde vende tilbage til Herren . Vi erc af Naturen saa form Mede af Synden , saa forblindedc af nens Fader , saa overvceldede af <b>aandelig</b> Dorstheds at vi ikke see , hvor langt vi have forvildet os ud i ørkenen , ikke ftle tilfulde , hvor jammerfulde vi ere , ikke med Alvor betente , hvilket ftrgeligt <b>Maal</b> vi ile imFde ; vi lignc det ~ tabte Faar " og den ~ tabte Penning " , der ikke har Vevidsthed om , at de ere tabte . Men see atter her , hvilken uendelig stor Kjcerlighed vor trofaste Frelser har beviist os ! Som Hyrden efter det Faar , der havde forvildet sig nd i ørkenen , som <b>Qvinden</b> tcendtc et Lys , feiede Huset og med Flid efter den Penning , som hun havde tabt , saaledes har og ~ den gode Hyrde " med Flid efter os . Han iledc os imFde allerede i vort Livs tidligc Morgenstund ; han og fandt os ved den hellige Daab . Da bar han os hjem , idet han indlemmede os i den hellige , almindelige, christeligc Kirke , dette Fredens Hjem , hvor hans Naade rigeligen udstrMmer over os , hvor hans Kjcerlighed lyser for vor Sjcel og opvarmer vort Hjerte , hvor hans <b>Aand</b> ved Ordet og Sacramenterne sin Naadesgjerning i vort Indre, og hvor vi kunne leve i vor Frelsers salige Samfund ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081303006'target=_>Besser, W.F., 1858, Lukas's Evangelium</a><h4>37</h4>V . 4. Paa det du kan lcrre at kjende den Lcrr < doms Vished , i hvilken du er bleven mundtlig underviist. Den Ler dom , i hvilken Theophilus var underviist, er deu samme , i hvilken vi fra Barndommen af cre nnderviste af vor <b>Moder</b> , Kirken , faafremt den redelig har forvaltet mod os sit moderlige Embede . Jeg er en elendig fordsmt Synder og bliver uden Fortjeneste retferdiggjort ved den Forlosning , som er steet ved Jesum Christum , hvis egen jeg er ; dette er Lerdommen . den frelsende Christenlerdom , og denne <b>Lerdom</b> har sikker Grnnd , som staaer nbevegelig , nåar Jord og Himmel forgaae . Hvad min Herre Christus har gjort for mig , det kan Ingen gjsre nskcet , hverken jeg sclv eller Verden eller Djevelen . „ Et lidet Ord kan ham selde , " — nemlig dette : Fuldbragt . Naar min <b>Sjels</b> Fiende vandt saadan Magt over mig , at jeg maatte give Slip paa dette Ord , da var jeg forloren ; men saa lenge jeg holder fast ved og stotter mig til , at min Guds og Frelsers Gjerninger visselig og sandelig ere stete til min Forlosning, saa lenge har jeg Trost og Hjelp imod al Anfegtelse. Dette er Forskjellen mcllem Christcnlerdommen og alle de gamle og nye Hedningernes Lerdommc : dc Christne troe , hvad deres Gnd har gjort og endnn gjor for dem , Hedningerne indbilde sig , hvad deres Guder ikke have gjort og aldrig gjore . Men saa <b>vis</b> den frelsende Lerdoms Grnnd end er , den bliver dog übeviist for den , som ikke kan fordrage den , som intet Hjerte har for den . Dette har ogsaa Lukas vel vidst ; det er aldrig faldet

<h4>392</h4>ledes , faaer soge sig en anden Christns . Denne Christns er sat til et Tegn , som imodsiges , til Fald og til Ovreisning; dette maa ogsaa visselig hans Lem , enbver Christen , vere for sin Troes og sit Ords Skyld . Man maa fordomme, bandlyse og forbande hans Lere og Tro som det verste Kjetteri , den verste Vildfarclsc og Daarstab . Hvor dette stcer , der steer der ham Ret . Skeer det ikke , da er hverken Christns , eller hans <b>Moder</b> , eller Simeon , eller Propheter , eller Tro . eller Evangelium , eller Christne der . Endnn en Trost giver dette lille Ord . Han siger : Det er et Tcgn , som mod siges , men dog ei omstsdes og tilintetgjores . Lad al Verden fordomme og forkjetre min Tro , men ham maa den dog lade mig beholde, ham kan den dog ei fratage mig . Den bringer det ikke videre med al sin Larmen og Rasen , end at den knn modsiger mig . Derefter falder den , og jeg staaer . Flnerne snrre ogsaa med deres Vinger og hvesse deres Neb , men udrette dog ikke mere end , at de tilfole Veggen, men de lade den nok staae . " Sml . Ap . G . 28 , 22. — Hvad Simeon her forndsigcr om sin Herre , at han stulde modsiges , det er i fuldt <b>Maal</b> blcven opfyldt paa hans Person ; iser Joh . Cap . 5 ss . meddeler os Efterrct » nincf derom . „ Hcm haver taalmodig lidt en saadan Modsigelse af Syndere " ( Hebr . 12 , 3. ) . Endelig have de korsfestet ham . Jesu Lidelser , siger Eimeom , stnlde trenge gjennem Marias <b>Sjel</b> som et Sverd . Christi Trengsler vare jo hendes Trengsler . Men da det , der strår efter siges , at mange Hjerters Tanker skulle aabenbares , ogsaa maa gaae paa Maria , saa hentyder vel den Hellig-Aand tillige paa den Troeskamp , som Maria havde at gjennemkjempe ved Siden af Moderkjerlighedens

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111500034'target=_>, 1865, Bibelen, eller Den hellige Skrift</a><h4>2009</h4>sslttl > 1 Kong . ? , ss . ) . saalcdes kunde vel den Hentydning , vi her finde til Tempelhallen . indeholde et Vink om . at dette Edikt ndgaar fra Gnd , som den Konge , der sclv dommer sit Folk . — Forbandelse o : Gnds Fordsmmelfcsdom , som forud udtalcs over cnhvcrsomhilst Overtrcrdcr af Loven Son , udgaar over det ganske Land . Meningen er ikke den , at Dmnmcn gjelder og stal ramme det bele Land , men at den stal blive brkjendt i hele Landet , forat Ingen stal kunne undstyldc sig med sin Uvidenhcd . Hver den , som stjcrler hver d > n , som svcrrger falskelig . Tnve og Menedere anfores her efempclvis , de Forste som Erempel paa Overtrcrdere af Lovens anden Tavle , de Sidste som Excmpel paa Overtredere af Lovens fsrste Tavle . Disse ssulde fnldkommen udryddes af det hellige Folk , saa der ikke bliver Spor tilbage af deres Boliger sV . 4 ) . Dermed er ogsaa Ud > ryddelscn af deres hele Slcrgt forbunden ; thi den Ugudelige faar ikke beholde nogen Arvclod i Landet . — Og Engelen — gik ud . For bande han vcrret ssjult , usynlig , nu aabenbaredc han sig synligen ; dette siges her . fordi han nu griber virksomt md i den Handling , som Synet fremstiller . Sml . 2. 3. Den samme Betydning " laabcnbare , <b>vise</b> sig ) har Ordet  gaa ud " iV . 6 og 9. — En Efa . Cfacn . naK Honieren Hebrceernes storste <b>Maal</b> for torre Varer somtrent 2 ^ norsk Skjeppe ) , er her valgt paa Grund af sin Form og Storrelse , Her at tcenke paa et falskt <b>Maal</b> eller ogsaa paa et fuldt Syndemaal . gaar neppe an , da Teften ikke indcholdcr nogcn Hentydning hverken t ' l det Ene eller det Andet . Deres Oiemcrrke . Hebr .  dcres Die . " Dette kan forklares - ^ deres Skikkelse , deres Udsecnde ; altsaa : dette er deres Udsceude i alt Landet sdct hele Land ) , saalcdcs se de ud ; denne Efa er et Villede paa dem . Men Billede paa hvem ? Dette kan ikke verre Andre end Tyvene og Menedcrne , som ere ncevnte i det Foregaaendc sV . 3. og 4 ) . Dog faar dette forst sin fulde Betydning , naar man ikke tcrnker paa Efaen alene , men tilligc — hvorom der tales i det Fslgkndc — paa hvad Cfaen indcholdcr . paa <b>Kvinden</b> , som er Villede paa Ugudeligheden IV . 7. 8. ) . Meningen bliver da : Efaen , som indcstuttrr Kviuden . Ugudeligbeden , er et Billede paa alle de Syn » der , som stulle udryddes . — Andre overscette ssom vor Bibelovcrs . ) : s , Dctte er deres Hensigt sdercs Strcebcn ) i hele Landet . " Meningen vilde da verre : Syndernes Strcrben gaaer ud paa at gjore sine Synders <b>Maal</b> fuldt og saaledes nedkalde over sig i fuldt <b>Maal</b> Gnds Straf . Men denne Fortolkning har uanscet meget Andet , iscer det mod sig , at Efaen her slet ikke er Straffcmiddel . men blot et Kar . hvori Ugudclighcden stal fores bort fra det hellige Land til det Land , hvor dens Hjem er . — Stedet er dunkelt . — Det folgende V . overscrttcs forssjeUigt . Cnten som vor Vibclovers . : der  oploftedes shcevcde sig ) et Centner Bly ; " Meningen er da , at den Blyssive , som laa over Efaens Aabning , hcevcde sig , forat Profeten kunde faa se , hvab der var i den . Eller ssaaledes Schmieder , se Indledningen til C . 5. ) :  der svcruede et Centner Bly . " Meningen er da ikkc . at Efacns Aab » ning for var tildcrkket af Blyet , men at Blyet fra forst af svcrvcde , ved Slden af Efaen . forat Profeten kunde have det ved Hcwndcn naarßhan skulde tildcrkke Aabningcn paa Efaen og saaledcs hindre Ugudeligheden fra at ? siippe ud , fsrcnd den var kommen til sit Bestem »

<h4>2444</h4>Bsger i det G . T . Men allerede de Uoverensstemmelser med den virkelige Historie , som sindes i 1. Cap . , bevise dens Ucegthed , hvad ogsaa Hieronimus har anerkjendt . Baruch , Son as Neria og Profeten leremias ' s Skriver , der sammen med denne drog til Wgypten ester Jerusalems Ddelceggelse ( ler . 43 , 6. 7. ) opholdt sig , fortelles der os stmx i Begyndelsen af denne Bog , i det ste Aar ester Templets Odeloeggelse i Babel og lceste der denne Bog for Kong lojachin og de svrige fangne Isder paa den Faste- og Bededag , som holdtes til Minde om Templets Ddeloeggelse . ( Denne Fastedag holdtes paa den 7 de Dag i den ste Maaned , paa hvilken Nebukadnezars Hcerfsrer Nebusar-Adan var kommen til Jerusalem , forat opbrcende Templet . Sml . ogsaa Sach . C . ? . ) . De Tilstedeværende samlede Penge og sendte dem tilligemed de hellige Kar , som Fienden havde bortfsrt til Babel , til Presten lojakim i Jerusalem . I disse Angivelser vrimler det af Anstod mod den virkelige Historie ; thi i det sdelagte Jerusalem var dengang ingen Prester og intet Tempel , og der ossredes ester Templets Ddelceggelse ikke i Jerusalem , men i Mizpa ( ler . 41 , 5. 6. ) ; de hellige Kar bleve fsrst meget sednere sendte til Jerusalem as Persekongen Cyrus ( EZr . i , 7 — 11. ) * ) . Sammen med Pengene , som skulde anvendes til Offere , sendte de Fangne ^ denne Bog af Baruch og befalede , at den skulde lceses i Templet paa Festdagene og Aarsfesterne ( Sabbathen og / Nymaanedagene ) . Denne historiske Indledning ( 1 , 1 — 14. ) er Bogens svageste Del ; af dens videre Indhold se vi , at den er skreven af ett Mand , der var fortrolig med Guds Ord og det G . T . ' s <b>Aand</b> , uden at vi dog ere istand til at bestemme den Tid , Bogen er affattet . Bogen indeholder forskjellige Afsnit , som ere knyttede til hverandre uden Forbindelse : 1 ) En Bodsbsn af det nedbsiede Folk under Fangenstabet ( C . 1 , 15 — 2 , 35. ) . 2 ) Bsn om Befrielse fra den velfortjente Straf ( 3 , 1 - 8. ) . 3 ) En Formaningstale til Israel om at soge den sande <b>Visdom</b> og omvende sig til den ( 3 , 8 — 4. 4. ) ^ ) . 4 ) En Trsstetale af Zion til hendes fangne Bsrn ( 4 , 5 — 29. ) . Med Zion menes her Jerusalem i <b>aandelig</b> Forstand , de Troendes . <b>Moder</b> , Jerusalem heroventil ( Gal . 4 , 26. ) . 5 ) Trestetale til selve Jerusalem ( 4 , 30 — 5 , 9. ) . 6 ) Det saakaldte leremias ' s Brev ( C . 6. ) . Dette Brev af leremias , som blev vedlagt i en ' Afskrift , stal have vceret en Skrivelse , som Profeten stilede til de Fangne , der bleve forte til Babel 2 ) , for alvorlig at advare dem mod Afgudstjenestens Daarstab og Vederstyggelig bed ( se iscer 6 , 43. ^ ) ) , og dertil er Skrivelsen — af hvem den nu end er — vel stikket .

<h4>2842</h4>Denne Bog kaldes Salomos Visdoms Bog , som om denne kongelige Vismand selv havde strevet den , forat undervise Fyrster og Regenter. Dog maa detce blot ansees for en digterist Indklcedning , uden at dermed er tilsigtet noget Bedrag , da der i Bogen , som er forfattet paa Grcefi , ikke er nogensomhelst Antydning til , at det har vceret Forfatteren magtpaaliggenoe at give det Skin af , at den var forfattet af Salomo . Saavel Sprog som Tanker lede os til at antage , at den er forfattet af en Isde , som levede i Wgypten under Ptolemceus Fyskon ( mellem 145 og 116 f . Chr . F . ) , som var fortrolig med den grceste Verdensvisdom og til Guds og hans Folks Pris og i Modscetning til Hedensiabets Forvlldelser viloe fremstille den sande <b>Visdom</b> , som udspringer af Gudsfrygt og Lydighet » mod Guds Lov . Medens Jesus Sirach, som levede i Palcestina , oprindelig blot strev for sine Landsmcend , er denne Bog anlagt paa itle alene at lceses af Isder , men ogsaa af Hedninger . Forfatterens Navn er übekjendt ; men vi ville dog ikke undlade at omtale en Gisning , som er fremsat af en nyere Lcerd , om at Bogen skulde vcere forfattet omkring 130 af den alezandrinste Isde Aristobulus , som var Lcerer for de to kongelige Brodre : Ptolemceus Filometer og Pt . Fysion . Vi sinde her det samme Visdomsbegreb , som i det G . T . ' s helllge Skrifter oq Jesu Sirachs <b>Visdom</b> . Sml . Hiob C . 28. Ordspr . C . 8. og 9 Snach 1 , 8. 6 , 19. 14 , 20. 21. 24 , 1. <b>Visdommen</b> tcenkes som et selvstcendigt Vcesen , som i sig eier Ertjendelsen af Grunden til oq <b>Maalet</b> for den hele Skabning , som har baade Vilje og Magt til ved hensigtsmccssige Midler at iilbrette Alt i - Overensstemmelse med GudS Plan og fore det til sit <b>Maal</b> . Ja , hele dens Vasen bestaar itke i noget Andet end i denne <b>vise</b> Erkjendelse og Handlen . I denne Betydning betragtes den nu baade i Forhold til Menneskene og til Gud . I Forhold til Menneskene er den ligesom en <b>Moder</b> og Ovdragerinde , ja ligesom en Brud , som hengiver sig i hellig Kjcerligheo til dem , som begjcere den , og meddeler dem sit Vcesen , saa de ved Retfcerdighed og Kydsthed naa til Ertjendelse af alle Ting og faa det evige Lw . I Forhold til Gud er den det fsrststabte Vcesen , som fuldfsrer Skabelsen og staar ham bi ial hans Gjerning . Den fremtrceder altsaa ligeoverfor Gud og Menneskene som en Person , som har et selvstcendigt Liv , og der tales oftere om den som en saadan selvstcendig Person . Men da paa den anden Side denne Personlighed ikte har noget andet Indhold end at vcere <b>vis</b> og gjsre <b>vis</b> , saa kan man slet ikte fastholde dens Personlighed , men denne svinder hen for os , saasnart vi ville fastholde

<h4>2923</h4>Verdensorden . Men baade Tegn og Undere stulle tjene m at aabcnbare den <b>Vise</b> , hvad Tider og Stunder der forestaar . Tider og Stunder .  Tid " angiver kun det bestemte Tidspunkt ,  Stund " sTime ) dette Tidspunkts Bestaffenhed ° Undertiden kan Tidens Bcffaffenhed vcere angivet , uden at Tidspunktet angives nsiagtigt . Saaledcs i de fleste Spaadomme hos Profeterne . — Men jeg var et Barn af god Art og havde faaet en god Sjcrl .  En god Sjcel . " fra jeg blev undfangen i Moders Liv . sSml . Jesu Sir . 1 , 12. Anm . ) . Men Jesus siger :  Ingen er god uden En . nemlig Gud " sMatth . 18 , 17. ) .  Uden Nogen fodes paany af Vand og <b>Aand</b> . kan han ikke se Guds Rige " sloh . 3 , 3. ) . Hvorledes kan da den opdigtede Salomo her sige , at han er fsdt med en god Sjcel ? Han bekjender jo selv , at han er et dsdeligt Menneskes ? . 1. ) , og ved . at Dsden er kommen ind i Verden ved Djcevelens Avind s2 , 24. ) . Hvorledes kan han da fra Fsdselen af havt en god d . e . syndefri Sjcel ! David , den historiske Salomos Fader , bevidner , at han er fodt af syndig Sced og at hans <b>Moder</b> har undfangct ham i Synd sPs . 51,7 . ) , og ser netop i denne Erkjendelse den skjulte Sandhed , den hemmelige <b>Visdom</b> sPs . 51 , 8. ) . Hvorledes skulde Davids Son have havt en ren god Sjcel fra Fodselen af ? Den opdigtede Salomo erklcrrer . at Forceldrenes Kydskhcd er af hsieste Betydning for Nornene ; men David og Bathscba , Salomos <b>Moder</b> , havde allerede for dennes Fsdsel avlet en Son i Hor . Hvor » ledes kan da den opdigtede Salomo sige , at den historiske Salomo har havt en god Sjcel fra Fsdselcn af ? Alt dette er ganske sandt . Men vi maa lcrgge Mcerke til : 1 ) at den gode Sjcel . som her tiUcegges Salomo som Barn , ikke er en Sjcel , som er fri for den almindelige Syndighed , men en Sjcel , hvori der af Guds Naade var medfodte krcch tige Anlcrg til hellig Strceben . saa han allerede tidligen folte sig hendragen til Gud ug <b>Visdommen</b> . Dette staar ikke til at negte ; thi 2 Sam . 12. 24 - 25. hedder oet :  Og hun sßathseba ) fsdte en Son , og han sDavid ) kaldte hans Navn Salomo ; men Herren elskede ham . Og han sendte Bud ved Nathan , Profeten , og kaldte hans Navn ledidjah sdcn Elffede , Guds Ven ) , fordi Herren < elskede hanl ) . " 2 ) Denne Salomo er et , omend ufnldkomment , Forbillede paa den fuldkonme Davids Son , og som saadant fremstilles og betragtes han ogsaa af Profetcrne . Saaledcs er han nu ogsaa her et Forbillede paa den sande Davids Son , og derfor skildres han her uden de Pletter , som virkelig fandtes hos den historiske Salomo . — Det Vidncsbyrd om en god Sjcrl . som her gives Salomo , grunder sig , som sagt , forsaavidt Forvillede ! maa have et historisk Tilknytningspunkt , paa 2 Sam . 12 , 24 — 25. Mange nyere Theologer mcne vistnok , at Forfatteren her er inde paa den platoniske Lcere om Sjcelenes Prceexistens , og at det er hans Mening , at Sjcelen har renset og gjort sig god under et tidligere Liv her paa Jorden . Men da kunde det ikke hedde . at han havde faaet en god Sjcel . men at han havde havt , medbragt eller erhvervet sig en god Sjcel * ) . Ifolge Skriften beror derimod Salomos

<h4>3134</h4>rus . Thi Albinus havde dog idetmindste saamegen Skamfølelse , at han flvede sine Forbrydelser i det Skjulte ; men Florus br ^ d sig ikke engang om at skjule sine Skjendselsgjerninger , han gjorde sig til deraf , og behandlede Folket som om Keiseren ikke havde sendt ham forat styre det , men forat Fve Skarpretteremlcdet paa dem som dMite Misdedere . At plyndre Enkelte- tyktes ham et kjedsommeligt Arbeide ; han plyndrede helst Byer og hele Distrikter , og lod Røverne have fuld Frihed , naar de blot gav ham Andel i Nyttet . Hele Cgne forvandledes til Arkener , og en stor Mcrngde Icher udvandrede til Nabolandene . Ccstius Gllllus var dengang Statholder i Syrien . Da man var sig fuldt bevidst , hvor vanskcligt det var at faa Ret mod Florus hos de romerske Embedsmamd , da der stode samnange Midler til hans Raadighed , hvorved han kunde fordreie Sandheden , vovede Ingen at anklage Florus for hans Overordnede. Men da Gallus i Aaret 65 kom til Jerusalem til Paastefesten , glk en stor Skare paa flere Tusinde ham im ^ de og bad ham forbarme sig over Folket og beskytte dem mod den rasende Florus . Denne stod under Forhandlingen ved Siden as Gallus og sogte at gjpre Indernes Klager latterlige , men Gallus beroligede Folket og lovede at ftrge for , at Florus for Fremtiden regjerede med mere Maadehold . Derpaa vendte han tilbage til Antwchm , og Florus , som ledsagede ham til Kcesarcra , vidste underveis ved sine LMne og Bagvaskelser at indtage ham mod Isderne . Fra nu as synes Florus at have havt den bestemte Plan for Die , at ophidse Ipderne til Krig mod Romerne i thi paa anden Maade . saa han ingen Udvei til at kunne skjule sine Voldsgjerninger, men paa denne Maade kunde han maaske faa en crrefuld Plads blandt de store Hcrrftrere og faa Del i Byttet . . Det var dengang en meget tung Tid for Romerriget . Det var , som om Ondskabens og Fordervelsens <b>Maal</b> ikke alene var faldt og maatte gaa over i Palcestinll , men overalt ; det syntes som om alle Baand vare ovlMe . Man kunde tro , at Dommens Dag allerede na > rmede sig for hele Verden . Det er HM mcrrkvcerdigt , at paa samme Tid , som den jødiske Religions- og Statsforfatning gik tilgrunde , en Rystelse gik gjennem den hele Verden , som IlgnedZ den Solformørkelse og det Jordskjelv , som fandt Sted ved Jesu D ? d . Naar man betragter denne Tid , kan man verre i Uvished om man skal undre sig mest over den Frcekhed , hvormed Lasten fremtraadte , eller over Menneskenes Taalmod . Over det romerske Rige herssedc dengang Keiser Ncro . Han havde faaet en god Opdragelse , var udrustet med gode Anlceg , men var for tidlig , allerede i en Alder as 17 Aar , bleven Keiser ; hans skjendige <b>Moder</b> Agrlpvinas Herstesyge og en Har as Smigrere fordcrrvede ham snart ; Udsvwuelser og afstyclige Kvindercrnker ved hans Hof gjorde ham endelig til et grusomt , fillmlost Uhyre , som skred frem fra den ene Udaad til den anden , indtll han endelig lod sin egen <b>Moder</b> drcrbe . Dog nu , da den forferdelige Gjermng er fuldbragt , og Samvittigheden vaagnet , da han er i Nvished om , hvorledes han skal <b>vise</b> sig for sine Undersaatter efter en saadan Rcedselsdaad , da kommer det slaviste Folk og det endnu elendigere Senat ham festlig smykket imFde , de modtage ham som i Triumf ! Forat dove sin Samvittighed , styrter han sig i de uildeste Ndsvcevelser ; hans gamle Lc ? rer Sencka forlader Hoffet , og hans Yndlings tanke blot paa at tilfredsstille hans Lidenskaber og Luner , Han bliver mere og mere en Slave af alskens Vellyster , den utrolige Melhed gM ham mere og mere gridst , hans Luner forlede ham til de daarligste Handlinger; han udraaber : M mig har ingen Fyrste vidst alt det , han formaar — indtil han scrtter Ild paa Rom , gjennembryder Bjerge og mdenfor Knserboligen i en vandW Egn anlegger en Sp , som kunde rumme en Flaade , og Bjerge , Skove og Grcesgange , forsamler Senatet og forhandler for Alvor med det om en 2 Egtestabskontrakt mellem sig og tyende Mcend , ja indtil han endelig optrader offentlig som Sanger og Skuespiller , og drager omkring og beiler om den kunstelstende Mccngdcs Bifald i Rom og Grcekenland ! Hans Historieskriver Tacitus sinder det tilslut saa modbydeligt at berette om alle hans Grusomheder , at han siger ( 16 , 16. ) : Om jeg end fortalte om Krige blandt fremmede Folkeslag og om hvorledes romerske Borgere have offret Lwet for Fedrelandet , vilde jeg blive trcrt deraf og frygte for at trcette Lcrseren , som ikke havde Lyst til at lMe om hvorledes saamange Borgere , den ene efter den anden , have faaet en sprgelig , omeno ocrefuld Dch . Men hvor aands-

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008090903039'target=_>Lamartine, Alphonse de, 1847, Girondisternes Historie</a><h4>1921</h4>Saaledes talte Noland under den fsrste Blcendelse af Magten ; hans <b>Kone</b> horte ham med Vantroens Smiil om Loeberne ; hendes sastere Blik havde ved det fsrste Viekast seet en loengere Lobebane og et mere bestemt <b>Maal</b> , end denne forsagte og kortvarige Forligelighed imcllem et nedvcerdiget Kongedomme og en ufuldendt 3 icvolution . Det vilde have kostet hende for meget at frasige sig det Ideal , der stod for hendes glodende <b>Sjel</b> ; alle hendes Handlinger , alle hendes Ord , alle hendes Suk skulde , uden hendes Vidende , drive hcndes Mand og hendes Venner fremad til det .  Vogt Dig for Alles Trcedsthed og iscer for Din egen Dyd , " svarede hun den svagc og forfcengeligeßoland ;  Du lever i denne Hoffernes Verden , hvori Alt kun er Skin og de glatteste Overflader skjule de farligste Anslag . Du er kun en retskaffen Borger , som er forvildet ind iblandt Hofmcendene, en Dyd , som er udsat for Fare midt iblandt alle disse Laster . De tale vort Sprog og vi forstaae ikke deres : hvorledes ssulde de ikke kunne bedrage os ? Ludvig XVI , af en udartet Stamme , uden Op sving i Aanden , uden Kraft i Villien , har i sin Ungdom ladet sig kncevle af religisse Fordomme, som endnu sammenknuge hans <b>Aand</b> ; henreven af en letsindig Dronning , som med sit ssterrigste Overmod forener Beruselsen af sin Skjsnhed og sin hsieste Nang , og som af sit hemmelige og fordervede Hof gjor sine Vellysters Allerhelligste og sine Lastcrs Dyrkelse , holder denne Fyrste , forblindet paa den ene Side as Prcesterne og paa den anden

<h4>2364</h4>trykker sig uden Ovhsr frem , med et Spyd i Haanden , for at komme hen til Kongen . Det var den samme Morderknegt, som , to Aar tidligere , havde med sine Hcender afvadsket Berchiers og Foulons Hoveder i en Spand Vand og ved Haarene baaret dem til Ferraille-Quaien , hvor han tiltastede Folket dem , for at de skulde vcrre Blodbadets og Forbittrelsens Skilrter ved nye Myrderier . Et ungt lyshaaret Menneske , elegant kloedt , men med frygtelige Gebcerder , vedblev uophorligt at storme ind paa Grenadererne og stikke sine Fingre paa deres Bajonetter , for at skyde dem tilside og komme frem .  Sire , Sire ! " sireg han ,  jeg opfordrer Dem i Navnet af hundrede tusinde Sjele , som omringe mig , til at stadfcestc Forordningen imod Prcesterne ! dette eller , Doden ! " Andre Almuesmoend gjorde , endstjondt de havde blanke Sabler , Kaarder , Pistoler og Landscr , ikke nogen truende Bevcrgelse og ssgte at hindre Angrebene paa Kongens Liv . Man bemcerkede endog nogle Tegn paa LErbodighed og Smerte ide fleste Ansigter . I denne Nevolutionens Msnstl ring viste Folket sig frygteligt , men det gjorde ilke Sag med Morderne . En <b>vis</b> Orden begyndte at danne sig i Sålene ; naar Hob , trykket frem af Hob , havde betragtet Kongen og udstsdt sine Trusler i hans Yre , styrtede den ind i de andre Vcerelser og gjennemdrog triumferende dette Despotismens Palads . For at gjore Plads for sig jog Slagteren Legendre disse Horder af <b>Qvinder</b> og Born , som vare vante til at sijcelve ved hans Rost , foran sig . Han gjorde Tegn til at han vilde tale . Der blev Stilhed . Nationalgarden gjorde lidt Plads for at han kunde komme Kongen noermere .  Herre , " sagde han med en Tordenstemme og ved dette Ord , som indeholdt en Retsfortabelse for Kronen , gjorde Kongen en Vevoegelse , som udtrykte krcenket Vcerdighed .  la , Herre ! " gjentog Legendre med endnu stcerkere Eftcrtryk paa Titelen ,  hor os ! De er siabt til at hore os ! De er uden Tro og Love ! De har altid bedraget os ! De bedrager os endnu ! men tag Dem i 3 lgt , <b>Maalet</b> er fnldt ! Folket er kjed af at vcere Deres Legetsi og Offer ! " Efter disse truende Ord oplcrste Legendre en Petition , som var fuld as ligesaa bydende Udtryk og hvori han , i Folkets

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010021603016'target=_>Euripides, 1857, Euripides</a><h4>2803</h4>Hoist forstjcllig af Lyst og Hu Er vi Mennesker ; altid dog Sees hvad virkelig godt er . Ogsaa <b>Dannelsens</b> Magt er stor , Kraftig leder den frem til Dyd . Agt for Pligt er et Viisdoms Tegn , Og med Forstand at stue det Rette Hoster Tak , som er velfortjent , Naar omkring den Vcerdigcs Isse Wren snoer en uvisnelig Krands . Ivrig at strcrbe til Dyd er stort ; <b>Qvinden</b> virker i Stilhed kun , Mandens <b>Sjel</b> kan paa tusind Viis Prcege Dyden i skjsn Vedrift , Fedrelandet til Storhed og Glands .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092203005'target=_>, 1861, Almindelig Norsk Lovsamling</a><h4>1548</h4>saauidt de i dette Reglements § 9 omhandlede kenninger angaar . 3. Direktionen forestaar, efterat have paa den i fjerde Kapitel bestemte Maade foretaget Valg , selv sine Medlemmer for Kirke- og Undervisnings-Departementet , hvis Approbation paa Valget indhentes , forend den Valgte kan tåge Seede og Stemme i Direktionen . 4. Direktio « nen vcelger iblandt dens Medlemmer , Lcrrerne undtagne , hver Gang paa 3 Aar , en Foimaiid og en Viceformand , paa hvilke Valg ligeledes erhverves Kirke- og Undervisnings-Departementets Approbation . Bliver Nogen efter disse trende Aar atter valgt til Formand eller Viceformand , kan han undflaa fig for at modtage Valget . 2. Ligesom bet i Almindelighed paaligger Direktionen at paase og at udfgre Alt , hvad den maatte sinde paa nogen Maade at bidrage til at fremme Skolens <b>Formaal</b> , saaledes har i Scerdeleshed Formanden at vaage over , at Undervisningen , Sekretariatet , Regnskabet , Bibliotheket og Kunstskolens ovrige Indretninger holdes i beherig Orden , og at Skolens Lcrrere , Sekretcer og Betjente nsiagtigen udfere de dem paallggende Pligter ; i hvilket Biemed han , saa ofte han anser det nsduendigt , ber verre tilstede under Undervisningen, og efterse Archivet , Bibliotheket , Kunstsamlingen og Skolens ovrige Inventarium. G . Direktionen holder sine Msder ordentligvis paa en as de ssrfte Dage i huer Maaned , dog ikke i Undervisningstimerne ; og den samles i overordentlige <b>Moder</b> , saa ofte Formanden maatte anse det fornedent . ? . Enhver Sag , over hvilken Meningerne er ulige , afgjores , efter endt Diskussion , ved Votering , og med Undtagelse af de i fjerde Kapitel bestemte Tllfcelde , ved alminnelig Stemmefierhed , dog saaledes , at , naar Stemmerne er lige , Formandens tcelles som luende . 8. Direktionen skal erhverve Kirke- og Underuisnings-Departementets Resolution angaaende Anvendelsen af Summer , der ikke bestemt henhore til de reglementerede Udgifter , og angaaende enhver anden Sag afVigtighed. Dog maa , uden saadan Nesolutions Erhvervelse , en Sum af indtil 50 Spd . aarlig anvendes af Direktionen til ureglementeredc Udgifters Bestridelse , hvorom dog ved hvert Skoleaars Udlob Forklaring indsendes til Departementet . V . Lcerernes , Sekretceiens og Betjenternes Lon fastsettes af Departementet for Kirke- og Undervisningsvesenet, efterat Direktionen , overensstemmende med § 2 , derom har indgivet Forslag ( jfr. § 16 ) . lv Direktionen har , i det Seneste inben Udgangen af Juli Maaned gjennem Kirke- og Undervisnings-Departemeutet aarllgen at afgive en underdanigst Beretning om Skolens Virksomhed og okonomiste Forfatning for det forlobne Skoleaar , der hver Gang regnes fra 1 Juli i det ene Aar til 30 Juni i det selgende . « . Kunstskolens samtlige Protokoller autoriseres af Direktionens Formand , og forsynes med hans samt Tegne- og Kunstskolens Segl ( jfr. § 34 ) . R2 Breue og Erpeditioner underskrives af Direktionens samtlige Medlemmer , med Undtagelse af mindre vigtige Sager , der underskrives af Formanden alene . Alle Erpeditioner paraferes af Sekretæren . Tredie Kap . Om Lcererne . 53. Skolens Lcrrere anscrttes med 3 Maaneders Ovsigelse paa begge Sider . De fornodue Undervisnings-Assistenter anscrttes paa samme Maade og Vilkaar ; dog maa ingen Assistent autages , forinden Direktionen har indhentet vedkommende Lcerers Betcrnkning derover . Naar nogen Assistent anscrttes eller afsiediges, ber Direktionen derom gjere Indberetning til den Kongelige Norste Regjerings Departement for Kirke- og Undervisningsvesenet ( jfr. F 20 ) . < 4. Lcrrerne giue Undervisning i de for hver Klasse bestemte Grene af Kunsten , og efter den ved dette Reglement antagne Plan . t5 . De haue til den Ende at vcrre stadigen tilstede i Underuisnings-Timerne, og at vaage over , at Lcerlingerne med Flid benytte Undervisningen og <b>vise</b> et scrdeligt og semmeligt Forhold , samt at Betjentene opfylde de dem paaliggende Pligter i Klasserne . RV . De ere derhos pligtige til at mede i alle Direktionens Samlinger, undtagen forsaauldt Forhandlingerne angaa de i § 9 omhandlede Lsnninger . Ligeledes ere ^ de pligtige til , skiftevis at have Opsyn med dem , som udarbeide Prevetegninger paa Kunstssolen ( jfr. § 8 56 og 59 ) . R7 . De haue , hver i sin Klasse , Tilsyn med Inventariet , og tilstille Sekretceren Liste over Afgang og Tilgang i Skoleaaret ved scmimes Udgang ( jfr. 8 33 ) . tB . Naar nogen Lcrrer hindres fra selv at udfere Undervisningen, ber han derom gjere Anmeldelse for Direktionens Formand . Fjerve Kap . Om Optagelse af Medlemmer i Direktionen . < » . Medlemmer optageS enten ved at vcrlges til Lcrrere , eller efter Prove , eller uden faadan ( jfr. § 3 » . De tvende ferste Arter af Medlemmer have derom at henvende sig med skriftlig Ansogning til Direktionen . 2 tt . Ved Valget paa Lcrrere , hvorved Direktionen tåger Hensyn saavel til deres Duelighed og Indsigter i den Knnst eller Videnskab , i hvilken de agte at undervise , som til deres Evner til at lcere den fra sig , skulle i det Wndste tre Fjerdedele af Direktionens i Christiania sig opholdende Medlemmer vcere lilstede , og af disse udfordres en Stemmeflerhed af to Trediedele , for at den Sogende kan optages ( jfr , 8 13 ) . 2 « . Dersom den , der attraar at optages , er Kunstner , opgiver han i hvilken Gren af Kunsten han arbeider . Dlrektionen forelegger ham derpaa

<h4>1606</h4>afholdes . Han pllgter paa Opfordring eller af egen Drift at give Etablissementscheferne den fornedne Oplysning og Veiledning betreffende Skibsbyggeriet . ? . Han lader vedkommende Arbeidsforstander holde en Journal over det Mandstab , der dagllgen <b>moder</b> paa Arbeidsstederne , hvad Arbeide de have udfort , og hvad Materialier der ere forbrugte ; Rapvort herover indgiues til Vcerftschefcn saa ofte saadant maatte begjceres , og forsyner Konstruktoren eller Skibsbygningsinspekteren samme med sin Attest . 8. Naar der ifelge Vcerftschefens Samtykke eller Ordre arbeides paa Arbeidsstederne udenfor ben reglementerede Tid , skal han lade vedkommende Arbeidsfurstander herover forfatte en Beretning, som , forsynet med hans Attest , sendes Vcerftschefen , Naar han anser det hensigtsmcrssigt at afsiutte Akkord- eller Fursagt-Arbeide , indhenter han Vcrrftschefens Bestemmelse herom . V . Han meddeler aarlig til en <b>vis</b> Tid Vcrrftet en Opgave over de Materialier , som antages at burde anskaffes , og erholder til den Gnde af Vcrrftet en Opgave over de Arbeider , der antages at stulle udfores i Aarets Leb . < N . Han har Overopsyn med Konstruktionsssolen , og kan han , naar han anser det gavnligt , lade Eleverne drltage ide praktisse Arbeider paa de ham underlagte Arbeidssteder . Han er Medlem af Reguleringskommissionen og af Kommissionen for Haandvcerkernes <b>Dannelse</b> . z « . Han lader holde ' selgende Veger : l > ) Korrespondence-Protokol for udgaaende Breve ; b ) en Protokol indeholdende Ertrakt af indkomne Vreve ; c ) en do . indeholdende alle reglementarlste Bestemmelser Skibsbyggeriet vedkommende ; < ^ ) en do . , hvori sindes en Fortegnelse paa alle Marinen tilhørende Skibe ( saavel seilbare Skibe som Roftotillen ) med Rubrik for Skibenes Byggeplads , Tid , de leb af Stabelen eller vare nye , Tid , de sik mindre eller sterre Reparation , og hvor denne foretoges m . m . , hvad der kan tjene til at give fuldstcendig Oplysning om Flaadens Tilstand . Med disse Protokoller og fornedne Skrivematerialier forsynes han af Vcrrftet efter Nekvisition . R 2. Naar nogen Ofsicier bliver ansat hos hum . enten for at studere Konstruktionsfaget eller som Insvektionsofficier, da bliver han dennes ncrrmeste Foresatte , og lader ham forrette den Tjeneste, der maatte blive befalet , eller som han sinder gavnligst for Tjenesten .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112201009'target=_>Prytz, A.F.W.J., 1880, Livsbilleder fra Kristi Kirkes Historie</a><h4>370</h4>ydre Kaar bedrede » dog snart , da en rig Mand i Tagaste , Romanian , antog sig hendes Ssn paa en faderlig Maade . Augustin var imidlertid funken dybere og dybere i Lasten . 18 Aar gammel tog han fig en Bolerske , med hvem han levede fammen uden Giftermaal il 3 Aar . Hun fsdte ham en Ssn , hvem han — for Meblikket greven af en dybere Fslelfe — gav Navnet Adeodatus ( den af Gud Givne ) . Denne Tildragelfe gik ham dog ikke fynderlig til Hjerte ; han bekjender felv , at iet faadant fyndigt Samliv ere Born ikke velkomne . Augustin fslte fig ikke lykkelig i sit Syndeliv . Moderens Bsnner og Formaninger vare ikke ganske forgjcrves , og heller ikke var han istand til at glemme de mange fromme Indtryk fra den Tid , da hans gudfrygtige <b>Moder</b> fortalte fin lille Dreng om den Herre Jesus . For dette Navn havde han fremdeles en <b>vis</b> LErbsdighed . Hans Hjerte var uroligt ; det tsrstede efter ham , der ene kan fkjcrnke Menneskehjertet Fred . I denne Tid var det , at Augustin under sine Studeringer kom til at lcrfe en ( nu desvcrrre tabt ) Bog  Hortenfius " af den bersmte latinske Forfatter Cicero . Dette Skrift gjorde et fcerdeles stcerkt Indtryk paa ham . Det udtalte , at det var Menneskets Pligt ikke alene at soge og elske <b>Visdommen</b> , men ogsaa at sve den og leve for ' den . Hans Sjcel brcrndte efter det Evige og Himmelske , og han begyndte at stamme fig ved sit Liv .  Alle forfengelige Forhaabninger bleve mig pludfelig afstyelige " , vidner han ,  og med inderlig Hjertes Lcengfel snstede jeg at finde oen udsdelige <b>Visdom</b> " . Men Ciceros Bog kunde ikke yde ham , hvad han begjcerte . Han savnede ogsaa i dette Skrift lefu Navn , mcrrkeligt nok .  Mit fpcede Hjerte havde , Herre , indfuget dette min Frelfers , din Ssns , Navn med Modersmelken . — Hvad der var uden dette Navn , var det end nok faa videnstabeligt , tilpudset og sandt , kunde dog ikke begeistre mig " . — Nu vendte han sig ogsaa til den hellige Skrift  for at se , hvorledes det havde sig med den " . Men han blev skuffet .  Den syntes mig uvcrrdig , naar jeg fammenlignede den med Cicero . Min Opblcrsthed styede dens maadeholdne <b>Aand</b> , og min Forstands Skarpheo trcrngte ikke md i dens Indre . Det er dens Art , at den vokser med de Smaa . Men jeg foragtede at verre en af de Smaa , og opblcrst af Overmod tyktes jeg nng at verre stor " . I denne Tid , da han lcrngtede efter at begribe Guds Aabenbaring i Kristus Jesus , men tog Forargelse af Frelferens Tjenerskikkelfe , faldt han i Hcenderne paa en Sekt , der lovede at give ham , hvad han ssgte , nemlig Mcmikcrerne . Som Stifter af dette Kjcetteri ncrvnes en — forsvrigt temmelig sagnmcessig — Person ved Navn Mani . Denne Mand nedstammede fra en persisk Magerfamilie , var fenere gaaen over til Kristendommen og beklcedte en Presbyters Embede . Men saa fik han det Indfald at ville vcere en stor Reformator . Hans Lcrre er en storartet Sammenblanding af Kristendom og Hedendom , ifcer den faakaldte gammelpersifke Dualisme . Augustin fandt ikke sin Sjcrl tilfredsstiller ved , hvad han fik hos Manikcrerne; men han vedblev dog fin Forbindelfe med dem . Monika grced over Ssnnens Vildfarelfe ,  hedere Taarer , end en <b>Moder</b> kan grcrde over sit Barns legemlige Dsd " . Han vendte nu efter 5 Aars Fravcerelfe tilbage til fin Fcrdrestad og nedfatte sig der som Lcerer i Talekunsten . Men uagtet han glimrede ved sin <b>Begavelse</b> , var dog Moderhjertet uendelig bedrsvet ; hun vidste kun een 3 Ere for fin Ssn : at verre en lefu Kristi fande Discipel . I den selvsamme Stad levede de ncer og dog fjcrrnt fra hinanden . Thi i hellig Nidkjcerhed havde Moderen befalet fin Ssn , at han ikke mere stnlde komme for

<h4>555</h4>Daa samme Tid , som Tavler i Stratzburg holdt sin mcegtige Prcediken om Kristi fattige Levnets Efterfslgelse , levede ikke langt fra ham , i Konstans og , Koln , hans Ven og Ordensbroder , Dominikaneren Henrik S u s o . Omendstjsnt der er ikke liden Lighed mellem dem , var dog Suso en Mand af en ganske anden Art end Tavler . Medens denne optraadte som den mcegtige Prcedikant og som Saadan trak Verdens Blikke hen paa sig , flod SusosLw hen i Stilhed og var derfor ogsaa fattigt paa ydre Begivenheder . Klostrets begrcensede Rum var hans Verden , som han kun forlod for i forbarmende Kjcerlighed at gaa efter den Forlorne . Desuden manglede han den frie , evangeliske <b>Aand</b> , som Tavler var i Besiddelse af , og en evangelist Kristen vil finde Meget i Susos Liv , som vil vcere ham fremmed , og som han ikke tor tåge sig til Forvillede . Men dog er der faameget Elskeligt og Efterfslgelfesvcerdigt hos Suso , at en kort Skildring af hans Liv ikke bsr manale iblandt disfe kirkehistoriske Livsbilleder . Suso blev fsdt den 21 de September 1300 i Byen Kostnitz . Han nedftammede fra en gammel adelig Slcegt . Hans Fader var et Verdensbarn , streng oa af raa Sceder , som Riddersfolket almindelig var paa hans Tider . Hans <b>Moder</b> var derimod en from , elskelig <b>Kvinde</b> , der inderlig elskede Frelseren, og som taalmodig og freidig bar de Lidelser , som hendes Mgtestab bragte hende . Moderen har vel tidlig saaet i Barnehjertets Dyb den Sced , der senere bar saadan rig Frugt ; idetmindste vide vi , at han allerede i nittenaars Alder traadte md i et Kloster , der laa paa en af Bodensoens Ver . Ogsaa en Soster af ham gik i Kloster . Hans Overordnede sendte senere den rigt begavede Ingling til Dominikanerklostret i Koln , forat han skulde studere Theologi paa den derværende Hoiskole . Her var det , at der med Suso foregik den gjennemgribende Forandring, som gav hele hans senere Liv dets eiendommelige Retning . Hans Kristendom havde nemlig , som han selv fortceller , til den Tid kun vceret dod Form og Vane . Han var tilfreds , naar han i det Adre holdt sin Ordens Forskrifter oa ved fit Skriftemaal ikke havde at bekjende egentlig  grove Synder " . Men under dette fandt selvfolgelig hans Hjerte ingen Fred , og trods al ydre Fromhed var han fuld af dyb Uro . Da Horte han en Dag , da han fad tilbords i Klostret , forelcefe nogle af Salomons Ord om den himmelske <b>Visdom</b> , ved hvilken Herren grundfcestede Jorden , og hvorved Himlene bleve beredte , og forn giver den , der giver Agt

<h4>557</h4>paa den , et langt Liv og mange Aar at leve udi Fred , Mifkundhed og Sandhed, Naade og god Forstand for Guds og Menneskers Dine .  Hvad for en Kjcerlighed er dog dette ! " tcrnkte han ;  maatte den blive mig tildel , saa var jea vel faren ! " Og nu sluttede han en <b>aandelig</b> Kjcrrlighedspagt med Kristus , den himmelske <b>Visdom</b> , idet han lovede ham , at hans Hjerte og Lw flulde tjene ham i evig Trofkab . Suso var — som sagt — af en gammel ridderlig Familie , og over hans Personlighet ) er der ogsaa et vist nddermcessigt Prceg . Kristus , den himmelske <b>Visdom</b> , blev ligesom hans aandelige Kjcerlighed , hvem han bringer sin Hyldest , ligesom de verdslige Riddersfolk qjsre med  sme <b>Damer</b> " . Det er et barnliyt Spll , han drwer dermed , men fuldt af Ynde og Elskelighed , og det vil blwe forstå aet af den , der i sit eaet Hjerte har erfaret noget af den fsrste Kjcerligheds Ssdhed . Han lader sia paa Perqament male et Billede af sin Kjcrrlighed , Kristus , og stiller delte for stedse at have det for Vie i fin Celles Vindu . Naar Vcegteren forkynder den opgaaende Dag , saa falder han hurtig ned paa fine Knee og bringer sin Kjcerlighed sin Morgenhilsen . Ved hvert Berger Vin , som han drikker , beder han Jesus , sin himmelske Husvcrrt , til Gjcest . Fsrend han tåger paa sig et nyt Klcedningsstykke , anholder han fsrst om Jesu Velsignelse . Af Vaarens skjsnneste Blomster binder han sin Krans for dermed at bekranfe Billedet af Guds <b>Moder</b> med lesus-Barnet , idet han beder , at disse fsrste Blomster , ikke maa blive forsmaaede . Naar Mai Maaned kommer , er det Skik : Schwaben til LEre for denne Maaned at ovstille en grsn Gren . Han scetter Korsets Trce som sin Maigren , der er blomstrende af Naade , faaledes forn jo Maiarenene ere . For alle rode Roser bringer han sin Frelser hjertelig Tak , for alle smaa Fioler en ydmyg Neien , for alle yndefulde Liljer en mderlig Omfavnelfe , for alle de deilige Blomster , hvormed Sommeren smykker Skoa og Mark , en <b>aandelig</b> Kys , for al den liflige Fuglekvidder en usigelig Lovsang . Ja ofte er hans Hjerte faa fuldt af Guds Kjcerlighed , at han havde kunnet raabe op alle Mennesker , ja den ganske Skabning , Himmel og Jord , Luftens Fugle og Skogens Dyr , Solens Glans 09 de glinsende Dugdraaber paa Engens Grsnt til med liflig Strengeleg og Mende Toner at bringe den elskede Herre en Lovprisning uden Ende . Men Suso vidste ogsaa , at den , der elsker Herren maa lide med ham . Naar Samtidens verdslige Riddere droge ud for til sine Damers Behag at kjcempe og stride , — hvorledes skulde da han , der havde udkaaret stg den himmelske <b>Visdom</b> til sin Kjcrrlighed , kunne undgaa Lidelse og Smerte ? Og hans Tanke var fuldkommen rigtig . At vcere en Kristen og at kde , homger nsie sammen . Fsrst ved Korset bliver Egenviljen brudt , det gamle Mennesie med sine Lyster og Begjcerligheder dsdet , og det nye Menneske kan da daakg mere og mere fremkomme og opstaa , dette nye Menneske , der siger med Asaf :  Har jeg kun dig , Herre , da spsrger jeg hverken ester Himmel eller Erfaringen viser , at som oftest de stsrste Guds Tjenere ogsaa have vceret de stsrste Korsdragere . Den Kristnes Vei er Kristi Vei , og den fsrer gzennem Dsd til Liv . Men istedenfor at lade det vcere Herrens Sag at lcegge Korset paa , naar og hvor han vil , troede Suso , at han selv maatte paalcegge sia . alleslags  Kors " , en forkjert Mening , som dengang var meget almindelia 1 Kristenheden , og som hang sammen med mange andre falske Lcerdomme mden den katholste Kirke . Suso pinte og plagede sia selv paa den grusomste Maade , og han forvandlede bogstavelig talt den storste Del af sit Liv kl det bitreste Martyrium . Ingen evangelist Kristen vil falde paa at efterkgne hans selvvalgte Gudsdyrkelse ; men derfor bliver den Kjcerkghed tll Gud , der drev ham dertil , ikke mindre beundringsvcerdig og tillige befkjcemmende for

<h4>593</h4>Johan Hutz blev fsdt den 6 te Juli 1369 i den lille bshmiske Landsby Husinecz ncer ved den baierste Gramse . Der fortcelles os intet Ncermere om hans Barndom .  Vi kunne blot i Tanken forestille os Vondedrengens hjemlige Hytte i det skjsnne Bjergland , den stille Landsby , hvor de eneste Bygninger, der ragede op over de lave Hytter , vare Klostret og Nikolaus af Husinecz ' s Borg ; denne var Egnens Herre og Reformatorens senere Beskytter og Ven . Vist er det ifslge Hntz ' s egne Ord senere i hans Liv , at der var Intet i hans Opdragelse eller Omstcendigheder , der var beregnet paa athceve ham over den ydmyge Stilling , hvori han var fsdt ; men hans alvorlige , tankefulde Vcesen og hans Tsrst efter <b>Kundskab</b> viste allerede tidlig hans Forceldre , at Ssnnen ikke var noget almindeligt Barn . Han slugte snart al den <b>Lcerdom</b> , som det nærliggende Kloster kunde yde ham ; og da han forgjceves havde bsnfaldt de forfcerdede Munke om at lcere ham dedsdeSprog , som de selv Intet kjendte til , drog han for at fuldende sine Studier til Universitetet i Praschalitz . Der fordybede han sig i Studiet af Klassikerne med al den Iver , som et Menneske kan <b>vise</b> , der har naaet et lcenge forsnsket <b>Maal</b> , og ikke tilfreds med den Kundstabsmasse , han kunde samle ved et mindre Universitet , forfcerdede han sin <b>Moder</b> ved sin Hjemkomst med de Ord :  , , Med Guds Hjcelp vil jeg endnu bessge Praa . " " . I Prag , hvor Keiser Karl den Fjerde havde grundet det fsrste tyste Universitet , som dengang just stod i sin fulde Blomst og blev bessgt af mange tusinde Studenter , fortsatte altsaa den unge Hutz , dertil hjulpen af mcegtige Venner , sine i Praschalitz begyndte Studier . Han tog Magistergraden , blev udncevnt til Professor og snart efter Formand ( Dekanus ) for det theologiste Fakultet . Mere bekjendt blev han imidlertid , da han i 1402 blev kaldet til Prest ved Bethlehems-Kapellet , en Kirke i Prags Gamleby , forn en from bshmist Raadsherre , Johan af Milheim , havde bygget af egne Midler med den sceregne Bestemmelse , at der i den , istedenfor den latinske Messe , skulde blive prcediket paa Bshmist . Skarer af Tilhsrere strsmmede hver Ssndag til Bethlehems-Kapellet , deriblandt den fromme Dronning Sofie , der havde valgt Hutz til sin Skriftefader , og en stor Del af den fornemme Adel . Hutz var en scerdeles god Prcedikant . Hans Sprog var klart og overbevisende og ofte rent henrivende . Hans Prcrdikener vare strengt bibelske , hvad der dengang i Prag som overhovedet i Kristenheden var noget ganske Uhsrt ; med en Aabenhed , som ikke tog Hensyn hverken til det Ene eller Andet , revsede

<h4>866</h4>usedvanlige Styrke , der gjorde ham sterk nok til at tåge fat paa den Kjemveopgave , der nn forelaa ham . Fsrend vi imidlertid gaa over til at paavise , hvorledes Kalvin virkede for sin Opgave , maa vi betragte lidt nermere Kalvins Anstuelfer om Kirken og det Kirkeregimente , han fik indfsrt i Genf . Kalvin tilstod Kirken en stsrre Auktoritet end de andre Reformatorer . Han siger :  Kirken er vor <b>Moder</b> . Allerede denne enkelte Benevnelse viser , hvor nyttigt det er at lere den at kjende . Thi vi kunne ikke anderledes indgaa til Livet , end naar den fsder os af sit Skjsd , nerer os ved sin Barm og holder os under sit Opsyn og sin Beskyttelse , indtil vi , befrieoe fra dette dodelige Liv , blive Engle lige . " Denne Kirke har — lerer han videre — et af Gud indsat Lereembede , hvilket de Troende ere forpligtede til at <b>vise</b> Lydighed . Den har Ret til at optrede lerende . Synoden , som representerer den , og som bestaar af Geistlige og Mldste , yher den bedste Borgen for den rigtige Udleggelse af den hellige Skrift . Der tilkommer Kirken Ret til at give Love og haandheve Kirketugt . Intet Samfund kan beståa uden Orden og Tugt . Leren er Kirkens <b>Sjel</b> , Tugten er at sammenligne med Nerverne , som formidler Forbindelsen med de forskjellige Lemmer og holder denne Ordning fammen . Den er en Tsile for de Onde , en Spore for de Gode , et faderligt Ris . Den gjennemlsber forskjellige Grader , fra Privatformaning til offentlig Anklage og til Udelukkelse fra den hellige Nadvere . Forat den kan vere retmessig , maa den ikke haandheves af de Geistlige alene . Den saaledes organiserede Kirke er i sit Vesen strengt forfkjellig fra Staten . Denne arbeider mod rent jordiske <b>Formaal</b> , medens Kirken streber mod et himmelsk <b>Maal</b> . Derfor har den ingensomhelst Ret til at herske over Kirken eller til at blande sig ind i rent kirkelige Sager . Kalvin ssgte nu at benytte den Seier , han havde vundet , til at praktisere sine Theorier . Straks efter sin Ankomst erklerede han , at han ikke kunde bestyre sit Embede , uden at der blev oprettet et velordnet Presbyterium , forbundet med kirkelig Tugt . Derpaa gjorde han Udkast til en Kirkelovgivning , som den 20 de November i selvsamme Aar paa en hsitidelig Maade blev antaget af det samlede Borgerskab . Ved denne Lov blev Stadens kirkelige Inddeling fastsat , Antallet af G ei f t lige bestemt , deres Forretningskreds anvist og Valget af dem ordnet paa den Maade , at de bleve valgte af de andre Prester ; men dette Valg skulde bekreftes af Naadet og Menighederne . Ved Siden af Geistligheden anordnede han LEldste ( Presbytere ) , fom bleve udpegede af Presteskabet , men hvis Valg ligeledes blev stadfestet af Menighederne. Konfistoriet , en Domstol , der havde at vaage over Sedernes Renhed og at forvalte Kirkens Formue , blev valgt af og blandt Medlemmer af det borgelige Raad ( mod Kalvins Anste , der krevede det sammensat af Geistlige og sEldste ) . Til de forannevnte Menighedens Embedsmend , Prester og Mldste , kom endelig Di a <b>kone</b> r ne , Medhjelpere i Sjelesorgen blandt Fattige og Syge . Det var et jernhaardt Regimente , der nu blev lagt paa Genserne . Konsistoriemedlemmerne turde betrede hvilketsomhelst Hus til enhver Tid . Konfirmationen fandt Sted i det 16 de Aar ; Konfirmanderne bleve noie prsvede , forinden de stedtes til den fsrste Altergang . Ingen Daab skulde finde Sted i Husene , men i Kirken for den forsamlede Menighed . Konsistoriet havde Ret til at tale med ved Indgaaelsen af Mgtefkab . Der fandtes kun kirkelig Vielfe . Ingen turde klede sig  over sin Stand . " Den selflabelige Omgang blev sat sine Skranker : Antallet af Gjester og Retter blev bestemt !

<h4>887</h4>endnu var i Isdernes og Hednmgernes Hcender , at Bistopfterne og Fcrdrenc da havde faaet istand den Ordning , at der af Menigheden blev bestikket noglc celdre , redelige Mcend ( Presbytere ) , som flittig skulde givc Agt paa Kirkens Lemmer og formane dem , som levede forargelig , og , naar Formaningen M hjalp , under Biskoppens Forscede udelukke dem as Kirken . Da imidlertid den verdslige Magt var kommen i de Troendes Hcender , fandt han det bedst , at Avrigheden bestikkede nogle Personer af Borgerflabet til sammen med Presten at formane Lovovertrcederen . Det biskoppelige Ban havde hidtil mer ? vceret en Tilladelse til Synd end en Straf ; derfor var det fornsdent at ordne det saaledes , at Synderen ved Unddragelsen af de kirkelige Goder kom til Bevidfthed om , at han havde udelukket sig fra de Helliges Samfund , men ogsaa blev forsikret om , at Menigheden ikke hsrte op med at bede for ham . Hvad det tredje Spsrgsmaal , om Gudstjenesten , angaar , saa ordnede han den saaledes , at der istedenfor den katholste Morgenmesse paa Hverdagene skulde trcede en kort Bibellcesning og Sang , istedenfor den katholste Eftermiddagsgudstjeneste,  hvor man fandt Folk mere sovende end vaagende , " en liden Aftengudstjeneste med Salme og en ganske kort Prcediken . Ssn- og Helligdagsgudstjenesten lod han begynde med Prcediken , derpaa en Salme , saa , Fyrie eleison " og , Mre vcere Gud i det Hsie ; " herefter fulgte Evangeliet , Troesbekjendelsen , Nadversformaning , Nadverelementernes Konfekration og Uddeling , Slutningsformaning og Velsignelsen . Den , der vil sammenligne Luthers tyste Messe hermed , vil faa se , hvor ganske selvstcrndig Brenz er gaaen frem , og hvor stor Overensstemmelse der dog alligevel er mellem ham og Luther . Man kan overhovedet kalde Brenz den schwabiske Luther . Thi han er ikke blot Luthers Elev og Medarbeider , han er til en <b>vis</b> Grad selvstcendig i den Maade , hvorvaa han anordner Kirkevcesenet , og hvad dertil hsrer , i det Enkelte , medens det selvfslgelig paa den anden Side maa erkjendes, at Brenz modtog det fsrste evangeliste Syn fra Luther , at han i dogmatisk Henseende helt igjennem var luthersk og maatte kaldes en luthersk Reformator . Efterat disse kirkelige Anliggender vare ordnede , rettede han ligesom Luther sin Opmcerksomhed paa Ungdommens Undervisning og udgav endnu fsr Luther en  Lcerebog i den kriftelige Tro , bestemt for Ungdommen i Hall i Schwaben " i 1527 ) . Denne Bog ' bestod af 1 ) En liden Katekisme paa 9 Sider . Den begynder med Daaben ; derpaa fslge Troen , Loven , Bonnen , Syndsforladelsen og tilsidst Nadveren . 2 ) En stsrre Katekisme paa 34 Sider for Voksne . — Allerede i 1529 var denne den evangeliste Kirkes celdste Katekisme oversat paa Latin . Brenz udtalte i Fortalen til en senere ( 1535 ) udkommen storre Katekisme , at der var Meget i hine tidligere Katekismer , som han ikke nu lcengere kunde billige . Imidlertid udmcerke de sig ved barnlig Enfold og Varme og vare til rig Velsignelse . Med stort Alvor drev han nu paa , at man skulde ovre.tte bedre Skoler for Fattige og Rige , Gutter og <b>Piger</b> .  Ungdommen er et Borgerstabs hsieste Skat ; en from og dygtig Borger kan afvende mere Ondt fra en By end hundrede Gevcerer . " Og ved sin besindige , nsgterne , alvorlige og dog milde Optrcrden bragte han det dertil , at Raadet i alle Stykker fulgte hans Anvisning; det havde heller ingen Grund til at angre derpaa . Brenz var ikke blot en fortrinlig Overhyrde derved , at han forstod at ordne Meniahederne paa en hensigtsmcrssig Maade og at ssrge for dem med Ord og Sakramente , men ogsaa derved , at han formaaede med stor Skarpsindighed og dyb <b>Lcerdom</b> at afvcerge og bekjcempe al falsk Lcere , som vilde trcenge sig md i de nye evangeliske Menigheder , ja deri laa ret egentlig

<h4>901</h4>som lagde sit 3 Eg i Ncrrheden af hans Smuthul . Dermed opholdt han Livet , til Soldaterne vare dragne bort , og Faren saaledes var over . Nu begav han sig til Borgen Hornberg paa Schwarzwald , hvor han var et helt Aar . Hssten 1550 kaldte Hertug Ulrik ham til Urach , hvor hans Kollega Isenmann imidlertid var bleven Sogneprest ; da han doq heller ikke her var sikker nok , tilbragte han noaen Tid i det afsidesliggende Megerkingen; i September 1550 giftede han sig med Isenmanns Datter Katharina og levede med hende i det lykkeligste Mgteskab lige til sin Dsd . Dermed havde Trengselstiden sin Ende , og der begyndte et nyt Afsnit af Brenz ' s Liv . I det samme Aar dsde Hertug Ulrik , og hans Son Kristofer , en Discipel og ivrig Beundrer af Brenz og tillige en kraftig , tapper Mand , fulgte ham i Regjeringen . Han kaldte straks Brenz til sin Hoved stad , og Schwaber-Reformatorm blev atter rykket md i en riq , betydnings fuld Virksomhed . Der forhandledes dengang om de Evangeliskes Deltagelse i Koncilet i Trient , som efter en lang Stans igjen skulde aabnes 1557. Brenz ud ^ talte , at det vistnok var at forudse , at det vilde blive et Forhsr , som da Kaifas forhsrte Kristus , eller Ipperstepresterne forhsrte Peter og de svrige Apostle . Men da Undladelse af at msde frem kunde udlcegges som Foragt , Ulydighed eller Feighet ) , saa burde man skikke Afsendinge afsted ; dog burde man forbinde en frimodig Bekjendelse af den rene Lcere med en ligefrem Protest . Han udarbeidede selv med 10 andre lutherske Theologer den saakaldte ( ^ onfe3sio ( wyrtembergste Bekjendelse ) . Nogle Udsendinge bleve skikkede til Trient med Konfessionen ' ; men de Theologer , som fulgte efter , bleve ikke hsrte , da  man ikke vilde modtage nogen Undervisning af dem , forn vare skyldige at <b>vise</b> Fedrene Lydighet » . " Et nyt , i Marts 1552 afsendt Gesandtstab , hvoriblandt Brenz selv , blev opholdt i Trient lige til April Maaned , da det med uforrettet Sag maatte vende hjem igjen . Kort efter blev Koncilet spredt til alle Kanter ved Moritz af Saksens uventede Krigstog ( se Side 253 ) . Da Keiser Karl den Femte ved Forliget i Passau ( 254 ) var bleven nodt til at indrsmme Proteftanterne Ret til at bevcege sig frit , tog Hertug KMo fer , der var i hsi Grad misfornsiet med de ved Interimet fremkaldte Tilstande, og som lige fra sin Regjerings Tiltredelse hemmelig og forsigtig havde ssgt at modarbeide dem , — med Kraft fat paa at rense den wyrtembergske Landskirke for alt paveligt Uvcefen . Den katholste Messe blev afskaffet , og de katholste og interimistiske Geistlige , som ikke vilde boie sig , bleve afsatte . Ved denne Tid dode Stiftsprovsten ved Hovedkirken i Stuttgart , og Vrenz blev udnevnt til dette Embede , som var Landets hoieste geistlige Verdighet » . Han fik nu ifslge sin Einbedsstilling den stsrstelndflydelse , og han benyttede denne til at ordne Kirken paa Guds Ords Grunt » . Vi ville omtale nogle af hans Anordninger . Det var ham serlig magtpaaliggende at faa uddannet gode Prester . Forat deres <b>Dannelse</b> kunde blive grundig fra Begyndelfen af , og for at give übemidlede , men begavede Inglinge Adgang til at komme md i det geistlige Embede , forandrede han Munkeklostrene til Klosterskoler ( senere blev Tallet af disse Skoler indskrenket til fem . ) Videre blev der ved Hjelp af Klostrenes Midler i Tybingen anlagt en Stiftelse , der optog de Inglinge , der havde gjennemgaaet Klosterskolerne , til videre <b>Uddannelse</b> og ledede og vaaaede over deres Studeringer ved Universitetet .  For mange wsinde Wyrtembergere er denne Anstalt bleven en <b>aandelig</b> og legemlig nerende <b>Moder</b> . " — Ligesaa omhyggelig arbeidede Brenz i den skjonneste Forstaaelse med Landets Fyrste for Folkeskolen ; thi , som bekjendt , trives den daarlig i de

<h4>956</h4>Fromhedens <b>Aand</b> , der havde raadet i den saksiske Kurfyrstefamilie , Reformationens kraftige Ststte og Hjcelp , gik som en Arv i mange Sloegtled over paa Born og Barneborn og udfoldede sig til den skjonneste Blomst hos Johan Fredrik den Standhaftiges Barnebarn , Hertug Ernst den Fromme af Gotha . Denne Fyrste var vel ikke rig paa Lande og Folk , men for sine udmcerkede Gavers Skyld hsit anseet blandt Hsie og Lave , og Drivfjcrren nnder alle hans Gjerninger var en levende Gudsfrygt . Han blev fsdt den i Aaret 1609 som den niende Son af Hertug Johan af Weimar og hans Gemalinde Maria Dorothea . Da han allerede i sit fjerde Aar mistede sin Fader , faldt hans Opdragclse udelukkende paa Moderen , men var da i gode Homoer . Hendes trofaste Omsorg blev iscer velsignet for den unge Hertug Ernst . Allerede som Barn udtalte han iet os opbevaret Brev til sin  kjcere Fru <b>Moder</b> " den Bon , at han maatte faa en Bibel til Julegave , og hans Undervisning i Guds Ord var saa frugtbar , at han allerede i sit ellevte Aar kunde bede om at blive stedt til den hellige Nadvere , hvad der ogsaa blev ham indrommet . Da han derpaa traadte md i Dnglingealderen, anvendte han den storste Flid med at tilegne sig saadannc Knndskaber og Fcerdigheder , son : vilde vcere ham scerlig nyttige under hans fremtidige Regentvirksomhed . Han reiste omkring i Landet for at lcrre Forholdene at kjende , og han gjorde sig skriftlige Udtog af de Efterretninger , forn han fit , saaat han ved sin Regjeringstiltrcrdelse kunde have samlet ikke mindre end 1 < ) l > Bind med saadanne Udtog . Meget snart fandt han Leilighed til at bevise sine fortrinlige , ved omhyggelig <b>Uddannelse</b> Pleiede Gaver . De weimarske Lande bleve forvaltede ved en Landdirektion under den « loste Broder , Hertug Vilhelm . Hertug Ernst blev imidlertid for sin tidlige Dygtigheds Skyld trukken med md i denne Styrelse og maatte endog i sin Vroders Fravcer forestaa Landdirektionen . Den davirrende Tid var dog ikke faadan , at man udelukkende kunde vie sig til Fredens Sysler . Den ulykkelige Trcdiveaarskrig ( 1618 — 1648 ) havde allerede varet flere Aar , da i Aaret 1630 deu svenske Heltekonge Gustav Adolf kom sine . betrcengte Troesbrodre i Tyskland til Hjcelp . Da greb ogsaa Hertug Ernst til Svcerdet , idet han som Oberst traadte md i den svenske Hcer , og nu fik han god Leilighed til at <b>vise</b> , at en Fredens Mand ogsaa kan verre en tapper Kriger . Han tog med Udmcrrkelse Del i mange svcere Kampe og erhvervede sig scerlig i Slaget ved Lytzen , i hvilket Gustav Adolf fandt Heltedoden , en stor Fortjeneste af Hceren ved med den Del af samme , som han kommanderede , at gjore den tapreste Modstand mod General Pappenheims friste Tropper , faa den svenske Hcer beholdt Pladsen . Hvad Gudfrygtighedens Ovelse angik , saa var Hertug Ernst den Samme i Leiren som i Hjemmet . Hans Samtidige bevidne , at han midt under Trommers og Trompeters Larm ikke nndlod at have sine regelmcessige Andagtstimer , og at hans Telt var et

<h4>971</h4>raadede , og saa maatte den Sangergave , Gud havde givet ham , lcrgge deilige Ord paa hans Lceber . Allerede i 1648 blev nemlig et ikke ringe Antal af de gerhardtske Salmer offentliggjorte i en af Kantor og Musikdirektsr ved St . ' Nikolai-Kirken , Johan Kryger , foranstaltet forsget Udgave af hans « I ' r2xi3 piewtiZ meiica ( Gudsfrygts Ovelfe ved Sang ) . " Gerhardt var ogsaa ellers virkfom forn Prcedikant i Byens Kirke , naar der paa Grund af Ledighet » blev Anledning dertil , og han greb med Glcede enhver faadan Leilighet » . Herved blev han vel kjendt i Byen ; thi Alle bleve opbyga , ede af hans paa een Tid kraftige , alvorlige og dog faa trssterige , af aandekg Balsom dryppende Prcedikener. Og dog — faa gjerne han snstede at faa trcede md i geistlig Virksomhed — maatte han dog vente en rum Tid , inden det behagede Gud at anvife ham en egen Vingaardsmark . Herrens behagelige Time kom endelig . Den 13 de Marts var Provst Kasper Gsde i Mittenwalde , en liden By 4 Mil fra Berlin , afgaaen ved Dsden . Stadens Raad henvendte sig nu i en Skrivelfe til Mimsteriet i Berlin med Anmodning om at forestaa en Mand , forn maatte anfees stikket for det ledige Embede . Ministeriet bragte Paul Gerhardt i Forslag ; , / han er , " heder det i Skrivelsen ,  en Person , hvis Flid og <b>Uddannelse</b> er bekjendt , af en god <b>Aand</b> og en uforfalstet Tro , desuden af et cerekjcert og fredelstende Sind og af et kristeligt , ndadleligt Liv , hvorfor ogsaa Gerhardt er agtet og elstet af Hsie og Lave i Byen ; og der maa gives ham det Vidnesbyrd , at han paa vor venlige Anmodning gjentagne Gange med sine af Gud modtagne Gaver har gjort sig elstet og fortjent af vor Kirke " Paa dette gunstige Vidnesbyrd blev da Gerhardt kaldet til den omtalte By og modtog den 18 de November 1651 i Nikolai-Kirken Kirkens Indvielse til sit hellige Embede som Provst i Mittenwalde . Han aflagde - da i Ordinationsbogen fslgende Bekjendelse :  I den hellige , udelte Treenigheds Navn . Jeg bekjender og bevidner herved , at den Lcere , som indeholdes i den fsrste og « forandrede augsburgske Bekjendelse og dens Apologi , i de schmalkaldiste Artikler , begge Katekismer og Konkordieformelen , hviler paa den profetiske og apostoliske Skrifts klareste og fasteste Grund , — og at jeg ved Guds naadige Bistand vil forblive hart ved samme indtil mit Livs Ende . " Dg at Gerhardt har holdt dette Lsfte , det er blevet vitterligt overalt , hvor hans Navn kjendes . I Begyndelsen af Aaret 1652 tiltraadte han fit Embede i Mittenwalde . I hvilken <b>Aand</b> han rsgtede sin Gjerning , kunne vi lettelig tcrnke os , stjsnt der mangler os ncermere Underretning om hans pastorale Virksomhed dersteds . Men Korset skulde han ogsaa her faa bcere . Anden Prest i Mittenwalde , Allborn , kunde ikke tilgive Gerhardt , at denne havde faaet Provsteembedet i Byen , medens han selv var bleven forbigaaet . Som let kan skjsnnes , maatte et saadant Forhold volde mangeslags Übehageligheter . Gerhardt var kun fem Aar i Mittenwalde . I det sidste af disse hentede han hjem som sin kjcere <b>Hustru</b> Datter as sin faderlige Ven Bertholdt i Berlin , Anna Maria , og han kunde nu fremvise for sin Menighed som et Monster til Efterfslgelfe Billedet af et kristeligt Hjem . Gerhardt fslte sig meget lykkelig i Besiddelsen af sin trofaste Mgtefcelle , hvad der ogsaa fik sit Udtryk ien meget vakker Sang . Den 19 de Mai 1656 blev deres fsrste Barn , en liden Datter , Maria Elisabeth , fsdt , men dsde , inden hun havde fyldt Aaret . Det Samme var Tilfceldet med i det Hele fire af de fem Born , som

<h4>1033</h4>op og med lydelig Stemme og under Taarer at raabe til Gud . Kristian Skriver erindrede endnu som gammel Mand , hvorledes han kunde vcrrevaagnet op og da fik hsre paa sin Moders brcendende Bonner . Han kunde mindes , at hun havde bedet for ham med omtrent fslgende Ord : „ Velsign og udrust ham , o Gud ! med <b>Visdom</b> og Forstands <b>Aand</b> , med Erkjendelses <b>Aand</b> og Herrens Frygt ! Velsign hans Studeringer ! Giv ham de nsdvendige Midler dertil ! Bevar ham for Djcrvelens Magt og List , for Verdens Forargelse og Synder ! Gjsr ham til din Naades Kar ! Lad ham vcrrdiges i sin Tid at tjene dig i din Kirke ! Benyt ham til din LEres Fremme og mange Sjceles Opbyggelse og gjsr ham endelig evia salig ! " Den Fader eller <b>Moder</b> , der kan bede saaledes og det as Hjertet for sit Barn , har lagt de signende Hcender over det , har opfyldt sin storste og bedste Fader- og Moderpligt . Den stakkels Enke havde ikke nok til at leve as i Byen og maatte derfor flytte ud paa Landet , og der var liden Udsigt til , at vor Kristian skulde faa uyde den Undervisning , som var nodvendig , om han skulde kunne blive Prest , saaledes som de dsde Fcrdre og Moderen saa inderlig snskede . Men da kom hun til at huske paa en rig Onkel , Kjsbmand Thomas Hebbers i Lybeck . Hun reiste til ham med sin Kristian og bad ham om , at han maatte antagc sig den faderlose Gut og hjcelpe ham frem . Kjobmanden lod anstille en Prove af den niaarige Drengs <b>Kundskaber</b> , som befandtes at vcere ualmindelig store . Han fattede snart stor Godhed for den loervillige , begavede Gut .  Min Son , " jagde han til ham ,  frygt Gud og studer flittig og vcer lydig mod din <b>Moder</b> og dine Lcrrere . Jeg vil ssrge for dig , at du engang kan takke Gud og mig , naar jeg ligger i min Grav . " — Han holdt sit Loste , idet han aarlig under ^ stottede Kristian med 50 Thaler . Kr . Skriver vedblev nu at gaa paa Skolen i Rensburg til 1645 , da han paa Grund af Krigen drog til Lybeck ; her fuldendte han sin Skolegang og kom i 1647 til Universitetet i Rostock . Hans Velgjorer hebbers , der imidlertid var afgaaen ved Dsden , havde testamenteret ham rigelig nok til Opholdet ved Universitetet . Under sit Studium sluttede han sig iscer til Professor Lyt ke mann , der tillige var Prest i Byen . Denne Mand havde engang tiltraadt sit Embede med disse Ord :  leg vil hellere fsre een Sjcel til Saligheden end gjsre hundrede lcerde . " Da nu Skriver kalder ham sin  Lcerer og Skriftefader , " saa maa man for ^ mode , at han af Lytkemann er bleven fort baade til <b>Lcrrdom</b> og Fromhed , dog saaledes , at det at blive i Sandhed gudfryktig og from stod langt hoiere end at trcrnge dybt md i Lcrrdommen . At Skriver ikke alene bad ivrig , men ogsaa stnderede flittig , fremgaar deraf , at han efter kun 2 Aars Ophold ved Universitetet var istand til at holde en offentlig Disputation paa Latin over Herrens Nadvere ( 6 e cona Domini ) . I Aaret 1560 forlod han Universitetet og vendte tilbage til Holsten , hvor han tog en Huslcererpost i den lille By Segeberg . De fromme Indtryk , Skriver havde modtaget som Barn , vare blevne befcrstede under hans Opvcekst . Han levede et inderlig Bonneliv baade som Skolegut , Student og Kandidat ; derfor kan han ogsaa , idet han ser tilbage paa fit Liv , udbryde i denne herlige Lovprisning :

<h4>1037</h4>Under sin Virksomhed som Huslcerer erhvervede han sig ogsaa paa sine Venners Opfordring Magistertitelen . Og han var suldt vcerd at bcere dette Lærdommens Navn . Ved gode Evner og stor Flid havde han tilegnet sig saare mange Kundstaber , og hans mange Skrifter <b>vise</b> hans store <b>Lcerdom</b> i de forskjellige Videnskaber , baade som Theolog og Sprogmand , som Historiker og Retslcerd ; endog i Medicin og Naturvidenskab var han hjemme . Og hos ham sik man rigtig se , hvor godt det er at have Kundstaben , naar den kun bruges rettelig , nemlig til Guds LEre og Medmenneskers Vel , til Fremme af Alt , hvad der er skjsnt og cerbart . Han var ydmyg og enfoldig ; og under alle sine Forskninger paa de forskjellige Videnskabers Omraade var og blev dog Bibelen hans kjcereste Studium . Derfor blev han ogsaa en Skriftlærd , der soger sin Lige ; hans Skriftkundskab var overmaade stor , og som den lcerde Hebraiker og Grceker kunde han trcenge dybt md i Forstaaelsen af de hellige Skrifters Grundtekst og derfra hente mange Skatte . Saa stod da den unge , begavede , baade fra Hjertets og Forstandens Side rigt udrustede Mand der , ventende paa , at Herren skulde kalde ham til Arbeidet i sin Vingaard . Herren havde ogsaa en Plads for ham . Den 2 den August 1652 blev Skrivers Halvssster , Lucie Kyhlmann , gift med Kristofer Trincius , Konrektor ved den lcerde Skole i Stendal i Altmark . Paa sin Moders Ynske fulgte den unge Kandidat sin Halvssster didhen . Han Prcedikede der flere Gange til stor Glcede for Stendalerne , der bleve saa glade i ham , at de ved Trincius ' s Befordring fsrst vilde hverve ham for sin Skole . Men Gud havdc andre Tanker ; han fsiede det saaledes , at Skriver kort ester blev kaldet til Arkidiakon ved St . lakobs Kirke i Stendal . I den Prcediken , han holdt ved sin Indscettelse i Embedet , sagde han tilslut :  Den Herre Jesus give mig et freidigt Mod under alleslags Trcengsler , at jeg ikke agter Noget , ikke engang mit Liv for dyrebart mod det , at jeg med Glceoe kan fuldkomme mit Lsb og detEmbede , som jeg har modtaget af den Herre Jesus . " Og til dette fromme Ynske sagde Gud Herren Ia og Amen . Den 10 de Mai 1653 indgik Skrider i VEgteskab med Anna Margarethe, Datter af Superintendent , NaF . Joh . Stralius ; hun blev ham en trofast <b>Hustru</b> . Fjorten Aar blev Skriver i Stendal , en Tid , der vistnok medrette kunde bcere Overskriften : „ Meget Kors , megen Velsignelse ! "  Allerede i mit fsrste Embedsaar , " fortceller han ,  tog min Gud efter sit alvise Raad og Vilje min fsrste kjcere Wgtefcelle fra min Side . Hvad dette kostede af Taarer , vide nok de , som vare om mig . Oa.saa har jeg senerehen sendt fire Born foran mig til den salige Evighet » , hvilket heller ikke stede uden mange Taarer og megen Bedrsvelse . " Desuden havde han at kjcempe med Nceringssorger ; thi vistnok var nu Krigens egentlige Storm over ; men dens Fslger bleve mcerkbare endnu i lange Tider , oa vare Menighederne i daarlige Kaar , saa drog dette selvfslgelig ogsaa efter sig , at Presterne havde ondt for at faa sine Indtcegter , trods vrstnok Menighederno ihukom det apostoliste Ord om , at  den , som undervises i Ordet , stal dele alt Godt med dem , som undervise ham " . Men han har selv sagt :  Korset er en nsdvendig Ting i Prcedikeembedet,

<h4>1075</h4>Da Filip engang som tolvaarsgammel Gut havde ladet sig sorlede til at tåge Del i Dans , blev han overfalden as en saadan Sjeleangst , at han ssgte hen til en skjult Krog og der i en heftig Strsm af Taarer lettede sit Hjerte , og under sit hele Liv beholdt han fra den Stund af den stsrste Modbydelighed for Dans ; han snskede endog senere , at der skulde udstedes lovmessige Forbud derimod . Han gjorde nemlig gjeldende , at denne Lsben og Hoppen omkring ikke passede sig for den Kristnes Wrbarhed og Verdighed — at det paalaa en Kristen at give Agt paa al sinGjsren og Laden , paa sin Tale , sine Geberder og sin Handlen , vel vidende , at han for alt dette engang skulde gjsre Regnskab . Femten Aar gammel kom Spener til Universitetet i Strasiburg og begyndte der at studere Theologi . Dr. Johan Rebhan , hans Onkel , tog ham i sit Hus . Den gamle Mand behandlede ham som sin Ssn . Spener tog fsrst fat paa alle Theologiens Forstudier . Han leste med Flid Hebraist og gjorde sig ogsaa bekjendtmed Arabisk . Til stor Velsignelse blev det ham , at han fra sin Barndom af var vant til en streng Helligholdelse af Ssndagen. Han asholdt sig ikke blot fra verdslige Forlystelser , men ogsaa fra alle lcerde Studier . Stoll havde nemlig alvorlig indprentet ham :  Om Ssndagen maa man Intet gjsre , hvorved man kan blive lcerdere , men kuns det , hvorved man kan blive bedre og srommere . " Efterat han havde del taget i den offentlige Gudstjeneste , leste han i Opbyggelsesbsger eller nedskrev Betragtninger ' Om den udholdende Flid , hvormed han allerede dengang drev paa sit Arbeide , forteller han senere : „ Engang vare omtrent seks Uger gaaede hen saaledes , at jeg tildels flere Dage itrek knapt saa etMennefie, ikke engang den <b>Pige</b> , som bragte mig Maden og fatte den udenfor min Dsr . " I 1654 blev han ansat som Lerer til to adelige Bsrn . Paa lohannes-Dagen 1655 holdt han sin fsrste Prediken over Ordene i Luk . 1 , 74. 75 - .  At vi , naar vi vare friede af vore Fienders Haand , skulde tjene ham uden Frygt i Hellighed og Retferdighed for ham alle vort Livs Dage . " Ta de to Grevessnner , hvis Opdrager han var , efter den daverende Ttd « Skit i 1056 droge til Paris , frasagde han sig sin Post , forat han ikke under de Adspredelser , hvortil en saadan Reise vilde give Anledning , skulde tåge Skade paa sin <b>Sjel</b> . Nsden drev ham nu til at holde private Forelesninger, og han leste over Filosofi , Geografi , Historie og Heraldik . De mange forskjellige Fag , hvori han var istand til at meddele Undervisning , <b>vise</b> os , hvilke omfattende <b>Kundskaber</b> han ved sine gode Evner i Forbindelse med sin lernflid havde veret istand til at erhverve sig . Meget vigtigt for Spener blev hans Ophold i Aaret 1659 baade i Basel , hvor han under Ledelse af den beromte Professor Buxdorff studerede videre i det hebraiske Sprog og terte sig det franfke og italienske Sprog , og iser i Genf . Her blev han fsrst holdt fast af en langvarig Sygdom ; men da han var bleven frisk igjen , sluttede han et inderligt Venstab med den reformerte Prest Anton Leger , af Fsdsel en Valdenser ( se Side 155 ) . Af denne fik han hsre , hvorledes Valdenserne levede oppe i Piemonts og Savoyens Fjelddale under et stadigt Tryk , ja ogsaa Forfslgelfe af Katholikerne , og hvorledes de valdensiske Predikanter ikke fik nogen Lsn , men levede af sine Henders Arbeide . Spener vandt under sit Ophold i Schweitz en saadan Kjerlighed for de franske forfnlgte Reformerte , at han senere altid mindedes dem med Kjerlighed og under deres Forfolgelser daglig bad for dem . Han traf i Schweitz ogsaa sammen med Jean de Labadie fra Frankrige , en Mand , der var opdragen af lesuiterne , men senere traadte over til den reformerte Kirke og sammen med en Anna Maria Schurmann Mede Laba

<h4>1084</h4>og at der var mange , mange Sjcele , som havde fslt Mangelen , og som med Glcede vilde gjsre Sit til for at raade Bod derpaa . Et stort Antal Prester saavel i hsiere som lavere Stilling udtalte sil : fulde Billigelse af , hvad der var udtalt i Speners ? ia 6 e3i6Sria ( Fromme Vnste ) , og bsde ham broderlig Haanden til fcelles Arbeide mod det store <b>Maal</b> , fremme af en livskraftig Kristendom . At der paa den anden Side udkom alskens Smcedestrifter og onde Rygtcr , var en Selvfslge , aldenstund Spener saa uforbeholdent gik los paa Skinkristendom og dsde Former , og for hans eget Vedkommende havde denne Forfslgelse den gode Virkning , at han gav mere Agt paa sig selv og gik forsigtigere frem , og saalangt fra at bersve ham Modet , gjorde det ham kun trsstigere . Saa blev han nu ogsaa glcrdet ved mangt et Budskab om , at den udftrsede Sced havde begyndt saa smaat at spire .  Ak , at det , som vi se her og der , maatte verre Skud , der bebude det snart kommende Foraar ! " skrev han da . Den nye Bevcegelse spredte sig omkring — fra Hus til Hus , fra Familie til Familie . Den greb Menneskene bande ide lave Hytter og i Slottene . Over det hele Tyskland gik der alskens Rygter om Spener og hans Lige sindede , — at de havde indfsrt fuldstcendigt Acrllesskab i Gods , — at i deres Forsamlinger baade Mamd og <b>Kvinder</b> prcedikede , — at alle Speners Til hcengere istedenfor den sedvanlige Dragt bare Linklceder , — at <b>Kvinderne</b> forssmte Husholdningen og i Sammenkomsterne sammen med sine Veninder fortalde alflens Lcetkerier , — at Spener var afsat fra sitEmbede — og lignende . Men det blev ikke kun ved disse onde Rygter . Speners Svoger , Horbius , blev afsat , fordi han hensynslsst prcrdikede og virkede i Speners <b>Aand</b> . Spener selv truedes med Anklage , fordi han roste mystiske Skrifter , der i sin Udtryksmaade fjcernede fig fra det orthodokse Sprog . Man paastod desuden , at han ivrede for Helliggjsrelsen paa Retfcerdiggjsrelsens Bekostning , og at han bersvede de Anfcegtede al Trsst . Men Spener vedblev rolig at arbeide videre uden at agte paa alt dette . Som et Middel , hvorved der kunde raades Bod Paa den herskende Ugudelighed blandt de Kristne , anbefalede han , at de alvorlige Menighedslemmer skulde slutte sig sammen til mindre Samfund , hvorfra da en Livets <b>Aand</b> vilde blcrse ud til de dsde Menighedslemmer . De Orthodokse havde kun tcenkt paa Bevarelsen af den reneLcere , de havde holdt Vagt paa Guds Stads Mure og Volde mod Angreb udenfra ; men de havde ikke tcrnkt paa den Pest og Hunger, der raadede indenfor Murene . Bekjendelsesskrifterne vare offentlige Troens Vidnesbyrd for Ven og Fiende ; men de vare ikke ufeilbare og ingen Troens Regel , der kunde scettes ved Siden af Skriften . Ligesaa fri stod han ligeoverfor Luther . Han kjendte Luthers Skrifter nsie , han satte dem meget hsit ; imidlertid var der dog ogsaa i dem Feil og Skrsbeligheder som i ethvert menne skeligt Vcrrk . — Spener var overmaade mild ligeoverfor andre Konfessioner . Guds Naaderige strakte fig vidt ud over den lutherske Kirke , omendskjsnt det stod fast , at intet andet Kirkesamfund kunde sammenlignes med den , hvad angik Lcerens nsie Overensstemmelse med Guds Ord . — Ligeoverfor Katholi kerne var han dog meget kraftia . i fine Udtalelser ; thi han fandt hos dem , hvad han scerlig havde meget imod : menneskeligt Magtsprog i Troessager , den vilkaarligste Forvanskning af de kristelige Sandheder og Gjerningshellighed i Livet . Han saa i det pavelige Rom Johs . Aabenbarings  Babel . " Hvad Paulus engang sagde om sia . selv , nemlig at han havde dagligt Overlsb og Bekymring for alle Memgheder , det kunde ogsaa passe paa Spener ; ogsaa om ham kunde det hede :  Hvem er skrsbelig , nden at ogsaa jeg er skrsbelig ? Hvem bliver forarget , uden at det brcrnder mig ? " Dertil kom

<h4>1088</h4>der — med Hensyn just til Mulighedeu af at komme til Dresden — - havde ytret :  Da du lsb med Fodgjcrngere , da gjorde de dig trcrtte , og hvorledes vilde du vove det med Hestenes ) " — blev rent bestyrtet , og en fuldstcendig Raadlsshed bemcrgtigede sig hans Sjcel . Om Aftenen den nwvnte Dag tog hans celdste Datter det nye Testamente for at flåa efter et Bibelsted til sin kjerre Fader . Da falder hendes Sine paa Apostlenes Gjerninger 7 , 1 :  Og han sagde til ham : Gak ud af dit Land og fra dinSlcrgt ug kom til det Land , som jeg vil <b>vise</b> dia " — og Begyndelsen af 10 de Vers :  Og udfriede ham af alle hans Tramgsler . " Efter lange , indre Kampe og megen Tvivl besluttede Spener sig endelig til at folge Kaldet og overtage Embedet , der paa den Tid betragtedes som en af de vigtigste geistlige Stillinger i den lutherske Kirke . ' Hans Afstedsprcediken var over 2 Petri 1 , 15 :  leg vil og gjore mig Flld for , at I efter min Bortgang altid kunne have , hvad der kan bringe eder dette i Erindring . " Hans Bortreise var en sand Hjertelidelse for Menigheden . Dag efter Dag gik han omkring for at trykke de Manges Hcrnder til Afsked . Han var paa den Tid 52 Aar gammel , og W Aar havde han virket i Frankfurt am Main . I den Alder og efter en saa laug Virketid paa eet Sted er man lidet skikket til at finde Behag i Forandring af Sted og Forholde . Paa sin Reise til Dresden , der vårede i 40 Dage , logerede Spener med sin Familie engang hos en Fyrste . Denne havde kun lidet Trosteligt at fortcrlle om Livet ved det saksiske Hof , men sor imidlertid at have Noget at troste den smertelig bevcrgede Prest med slaar han op Bibelen ; da falder hans Mne paa Sakarnis 4 , 7 :  Hvo er du , du store Berg ? du skal blive til en Slette for Surubabels Ansigt ; og han stal udfsre Houedhjornestenen , saa de stulle oplofte Rosten , raabendc : Naade , Naade vare nied den ! " Den nye Overhofprcedikant begyndte sin Prcediken med Matth . 5 , 20 — 20 , og paa hans nye Virkested fandtes der i Sandhed megen baade farisceisk Gjerningshellighed og aabenbar Lastefuldhed . Hoffolket med sit übundne , usoedelige Liv hadedc ham allerede paa Forhaand , og blandt Thcologerne i Dresden hersked ? den <b>Aand</b> , som han altid havde sogt at bekjcrmpe ' . Vi ncrvne et enkelt , meget betegnende Trcek : han aflagdc Visit hos alle sine geistlige Embedsbrodre , uien kuns een af dem gjengjeldte Visitten . Men han styrkede sig selv ved flittig Bon til i Taalmodrghed at holde ud .  leg har altid for Me , " sagde han selv ,  Ordene i Markus 4 , 28 : Thi af sig selv boerer Jorden Frugt , forst Grcrs , derefter Aks , derefter fuldkomnient Korn i Akset , og Jak . 5 , 7 : Se , Bonden forventer Jordens dyrebare Frugt og bier taalmodig efter den , indtil den faar tidlig Regn og sildig Regn . " Om Herrens Ord i 2 Tim . 2 , 24 — 26 :  Men en Herrens Tjener bor ikke stride , men vcere mild mod Alle , skikket til at lcrre Andre , istand til at taale Ondt , saa han med Sagtmodighed tugter dem , som modsEtte sig , om Gud engang vilde give dem Omvendelse til Sandheds Erkjendelse , og de kunde komme til sig selv igjen fra Djcevelens Snare , af hvem de ere fangne til at gjsre hans Vilje , " — om disse Ord sagde han :  Det vil blive min fornemste Lektion , som jeg med Guds Hjcrlp har at lcere . " Ttl en scrregen Velsignelse blev Sfteners Katekismuseksaminationer for Voksne , som han ogsaa fik istand i Dresden . Disse Katekisationer blevc forst afholdte hjemme i Huset , og der indfandt sig i Begyndelsen kun faa Folk , som begjcrrede at faa bedre <b>Kundskab</b> om de Ting , der hsre Guds Rige til . Han havde ncrrmcst tcenkt paa Mcrnd og <b>Kvinder</b> i yngre Alder , men snart

<h4>1099</h4>Disse Meninger fremholdt nu Spener , 09 ligeoverfor sine Embedsbrsdres Tviol om , hvorvidt de kunde blive i Kirkens Tjeneste , eller de nsovendig maatte forlade samme , udtalte han , at ester hans Formening var deres Forbliven i Kirkens Tjeneste selv under dens mest fordcervede Tilstand mere stemmende med den guddommelige LEre og Kjerlighed til Nesten end Udtredelse . Og naar Presterne tenkte paa den Fare , som der for deres egne Sjete kunde vcere ved at forblive i Kirkens Gjerning , saa bad Spener , at de maatte gjsre et ret dybt Indblik , i den hellige Pauli kjerlighedsfulde Hjerte , der han af Kjcerlighed til Brsdrene snstede at vcere en fra Kristus forbandet Ting ( Rom . 9 ) . Hans Forvillede skulde man efterfslge og idetmindste elste Brsdrene saameget , at naar man havde Anledning til at arbeide paa deres Frelse , og det ikke uden alt Haab , saa skulde man ikke paa Grund af den med Embedet forbundne Fare for sine egne Sjele unddrage sig for at benytte enhver saadan Anledning . Fremdeles fremholdt han adskillige Punkter til Provelse for dem , som stode i det geistlige Emvede . Han opfordrede til Prsvelse af den Hensigt , med hvilken Presterne havde valgt det theologiske Studium , hvorledes de havde drevet dette , hvorledes de havde levet i Studietiden , — hvorledes de havde erholdt sit Embede , og hvilken Hensigt de havde ved Tiltredelsen af samme , om de havde ssgt , hvad de selv og deres Kjod begjcerede , eller om de havdc ssgt dyrekjsbte Sjceles Frelse , — i hvilken <b>Aand</b> de havde rsgtet sin Gjer ning , om det var i Pauli <b>Aand</b> , der han siger :  leg vil ikke fordriste mig til at tale Noget om det , som Kristus ikke har udrettet ved mig , " om de havde ssgt Gud om hans <b>Aand</b> og Naade , om de havde sukket uden Ophsr til Herren for sine Menighet » er og raabt hjertelig til ham i Anledning af den hele Kirkes Nsd , — om de havde beflittet sig paa i sine Prcedikener at forkynde Guds Ord med Hjertets Overbevisning , saa de talte af Gud og for Gud , eller om de havde prediket med hoie Ord menneskelig <b>Visdom</b> for at <b>vise</b> sin Kunst og deri sat sin 3 Ere , tenkt ligesaameget paaVeltalen hedens Regler som paa det guddommelige Ords Grund , — om de overhovedet til <b>Maal</b> for sine Prcedikener havde sat Menneskenes Gunst og Kjcerlighed eller noget Jordisk eller Guds LEre og Nestens Opbyggelse , om de , naar Leilighed havde tilbudt sig , havde som Sjelessrgere bessgt sine Tilhsrere og anvendt med <b>Visdom</b> den tilfeldige Anledning , som Gud havde fendt dem , og , talt hos de Syge , hvad enten de vare kaldede eller ikke , saaledes , som disses Frelse udfordrede det , — om de fsrte et kristeligt Hus og levede vel med <b>Hustru</b> , Bsrn og Tyende , — om de ved Indkrevningen af den dem rettelig tilkommende Indtegt havde gaaet saaledes frem , at dr derved paadroge fig Mistanke for Gjerrighet » , Unoisomhed og Übarmhjertighet», — om de vare godgjsrende efter Evne , — om de venligen havde modtaget beskedne og velmente Advarsler og Tilrettevisninger af Embedsbrsdrc og Tilhsrere og , hvis der laa en Misforstaaelfe ti ! Grund , da havde bevist sin Uskyldighet » eller , hvis de fandt sig skyldige , rettet Feilene og taknemmelig anseet Paamindelsen som en Velgjerning , — om de havde levet i Fred og Endregtighed med sine Embedsbrsdre , saavidt Samvittigheten tillod det , osv. — Vi have fuld Grund til at tro , at den Mand , der skrev saaledes , selv svede sin Gjerning i den <b>Aand</b> , som lyser frem af hans Ord . Saa havde ogsaa Spener en saa udstrakt Indflydelfe , at hans Meninger fik Indpas saagodtsom overalt i det hele Tyskland . Han stod i en udbredt, velsignelsesrig Brevveksling ; i eet Aar besvarede han saaledes 62 ^ til ham rettede , Guds Riges Sag vedrsrende private Breve , og endda laa 200 übesvarede !

<h4>1234</h4>Til en anden Ven ytrede han :  leg vil loegge dig paa Jesu Hjerte ; men lad nn ogsaa denne Swnd tjene dig dertil , at du helt og holdent hengiver dia . til den kjoere Frelser og lige udholdende som den kanancriste <b>Kvinde</b> anraaber ham om Naade . Hvilken Lykke skal det ikke voere , naar vi engang kunne forlade Alt og have en naadig Gud i Kristus Jesus . " I de sidste Dage , fra den fsrste til tredje April , maatte han for Aandedrwttets Skyld sidde i en Lcenestol , boiende sig snart tilbake mod Stolen , snart frem mod en Pude , som laa paa et Bord . Under dlsse lange Timer kunde man mcrrke en sagte , uafiadelig Ynken , saa svoere vare hans Smerter . Dog horte man ham aldrig beklage sig ; hans <b>Aand</b> var freidig og glad ; han bestjcrftigede sig med at tcrnke paa , hvor saligt det skulde blive oppe hos Gud . Engang — og det var et af hans sidste Ord — fagde han :  O , du ftakkels , ilanselige Lazarus ! Og dog skammede Himlens Enale sia ikke ved at boere dig ! " Under al denne svoere Lidelse horte man — som sagt — ikke et utaalmodigt Ord af ham , ja mcerkede ikke engang en utaalmodig Mine . Dette , som ogsaa hans barnlige Fortrsstning paa Gud , som ved Lidelsen vilde lutre ham og gjsre ham stikket for sin himmelske Herlighet ) , var hans Omgivelser en inderlig Trsst . Ellers vilde det — som de sagde — have vceret uudholdeligt at se en elsket Vens saa svoere Lidelser . Den 2 den April ved Middagstid saa man , at <b>Maalet</b> for hans aarelange Lcengsel var ncert . Ssvnen blev dybere og dybere , saa man ved Midnatstid ikke mere kunde voekke ham . Han forblev sovende , til han Kl . 2 om Morgenen opgav sit sidste Suk . Dette var 3 dje April 1769 ; han var da 71 Aar 2 Maaneder og 4 Dage gammel . Og nu , kjcere Lcrser , lad det vcere en Bsn fra dit Hjerte , hvad en Ven saade i Indgangsbonnen til den Ligtale , som han holdt ved Gerh . Tersteegens Baare : Maatte hans bedende , mod dig , o Gud , rettede Gemyt , hans nnlde , dig hengivne Sind , hans blide <b>Aand</b> , hans Fornegtelse af sig selv og af Verden , hans blsoe , medlidende Hjerte , hans audfrygtige og hellige Vandel — maatte dette til dit Navns Forherligelse blwe for os et velsignet Minde , at ogsaa vi folge ham efter til den salige Evighed ! "

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011062212001'target=_>Landstad, M.B., 1880, Kirkesalmebog</a><h4>4332</h4>bevar Kongen og Landet med al kirkelia oa verdslig Vvrighed fra al Ulykke . Gud aive os Fred og Rolighed . Gud unde os Helbred og Sundhed . Gud naadigen velsigne vor ( Aaer og Eng ) Mad og Drikke , vor Kurv og vore Levmnger . O Herre , tcenk i Naade paa alle Syge og dem , som lide Angest og Pine . Giv dem en god Nat , send Lindring og Legedom send dem Trost og Hakv . Hjcrlp dem til at kunne vcere glade i Haabet , taalmodige i Trceng . selen og varagtige i Bsnnen , og lad deres Forlssningstime, som bie paa dig , ikte vcere langt borte . O Herre , vcrr mig arme Synder ( min Husbonde , <b>Hustru</b> , Foraldre , Bsrn ) min Slcrgt og Venner fjern og ncer og alle Mennesker naa ° dig for Jesu Kristi Skyld ! Vogt os med Hus og Hjem , Gods og Formue fra Ilds-Nsd , Vands- Vaade , Tyvehaand og alt andet Ondt , og lad ingen Ulykke komme inden Dsre . Der lcegges Dage , Uaer og Aar til min Alder , Vor Klokke flåar , vor Tid bortgaar — og Evigheden forestaar . O Herre , leer mig saa at t2lle mine Dage , at jeg Maa bekomme <b>Visdom</b> i Hjertet! Leer mig saaledes at bruge de dyre Nllllde- Timer , at jeg altid med Frtmodighed kan bie efter min Frastillelses Stund og forvente i Haa » bet Guds Bsrns herlige Frihed . Ak , Herre Jesu , kom , kom naadig , kom ieN god Stund ! I din Haand befaler jeg vaade Legem og <b>Aand</b> , du forlsste mig , Herre , du trofaste Gud . Amen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092403008'target=_>Besser, W.F., 1866, St. Petri Breve</a><h4>412</h4>Lengselen . De smaa Born strige , indtil de blive tilfreds ^ stillede ved Moderens Bryst . . . Haver jeg ikke sat og stillet min <b>Sjel</b> tilfreds som den , der er afvant hos sin <b>Moder</b> ? ja , som den , der er afvant , er min <b>Sjel</b> i mig " , synger Israel i en Pilgrimsscmg ( Ps . 131 , 2. ) . I den 119 de Psalme er den Hellig-Aand uudtsmmelig i Beskrivelsen af denne heftige Lengsel efter Guds Ord . I det 20 de Vers hedder det : . . Min <b>Sjel</b> er knust for Lengsel efter dine Rette altid " , og i det 28 de Vers : . . Min <b>Sjel</b> flyder hen for Bedrovelse ; opreis mig ved dit Ord . " Og denne Lengsel er det rette Symptom paa vor <b>Sjels</b> Suudhed . Saalenge Varnet er begjerligt efter Modermelken , kan man vere uden Frygt for dets unge , spede Liv ; men naar denne Kost ikke mere smager det , da er det farligt sygt . Prov dig herefter , du Guds Barn !  Et nyfodt Gudsbarn er begjerligt efter at modtage til Nering den Melk , der rinder udaf Ordet . Ved dette sode Livsens Ord gaacr det frimodig fremad i det Gode og styrkes i Aanden . " Et nyfodt Barn vil leve ; Kjerlighed til Livet , Driften til at voxe og trives er indplantet i dets Natur , og saaledes er ogsaa det nyfodte Gudsbarn inogivet den hellige Drift at voxe i det aandelige Liv , og vi ere begjerlige efter den aandelige , uforfalskede Melk , at vi ved den kunue voxe til Salighed , voxe til vor Troes <b>Maal</b> , nemlig Sjelenes Frelse ( Kap . 1 , 9. ) , hvori vi nu fryde og glede os i Haabet , hist i Stuen . Denne V oxen har Apostelen strax i sin forste Hilsen tilsnstet de adfpredte Udlendinge ( Kap . 1 , 2. ) , ligesom han da ogsaa tilslut udtaler det samme Haab med Hensyn til dem ( Kap . 5,10 . ) , og , som oftere , formaner

<h4>856</h4>Hjerte sagtmooigt , og hvad ere alle gyldne Hals- og Armbaand imod det stjonne Sagtmodighedens Baand , som Aanden legger om en lydig <b>Kvindes</b> Hjerte ? Milde , venlige Ord giver hun sin Mand og lader sig gjerne sige af ham , thi „ hnn har en Viinstoks Natur i sig , der lader sig boie og styre smukt , som Viingaardsmanden vil , med et Halmbaand ; " hun fordrister sig ikke til at vere selvklog og gjelde Noget uden Manden , hvem hun er givet til Medhjelp , ikke herskende , men stille vil hun vere ( 1 Tim . 2 , 12. ) . Men al Sagtmodighed og Stilhed , hvormed hun er prydet for sin Mand , kommer af , at hun er stille for Gud . Hun vil ikke Andet , end hvad der er Guds Villie med hende , at hun stal tjene fin Mand og sin Familie og dermed Christum ; <b>Kvinde</b> og intet Andet vil hnn vere , som <b>Kvinde</b> vil hun vandre til Himlen ; det er hendes enfoldige stille Dnste ; „ som Gud vil styredel , " den Tanke giver hende snart No , nåar Uro cmfegter hende . Stille , tilfreds i sit Livs Gud , med <b>Maalet</b> stadig for Die , gaacr hun sin Vei ; i sin Guds Huus og i sin Mands Huus kjender hun sit Kald , der blive hendes Fodder gjerne ; at hun maa vere „ huuslig " ( Tit . 2 , 5. ) og „ gudfrygtig " ( 1 Tim . 2 , 10. ) er hende alene om at gjore og hendes eneste Dnste . Hun kan begge Dele , ifore sig en kostbar Kledning , om saa stal vere , og pryde sig med en simpel Kledning ; Intet volder hende Uro ; hun kan vere hoi og tillige ringe , „ dannet " og meget „ enfoloig , " eret og übemerket , thi den Fredens <b>Aand</b> gjor hende i alle Tilfelde stille , ved hvilken hun har erholdt , hvad d ^ er er uforgjengeligt , og som lerer hende hverken med « rolig Angest at stye eller med urolig Lyst at begjere ,

<h4>1630</h4>den funde , med dydigt Levnet forbundne Erkjendelfe i Modsetning til dens sygelige Dobbeltgjengerste , som de falste Lerere havde bragt paa Bane . — Videnskab ( vilde han maastee i vor Tid sagt ) , hvis Berommelse er saa stor ! — Men i <b>Kundskaben</b> Afholdenhed . Gnostikernes Erkjendelfe gik fvcmger med alslags Uafholdenhed; de Christnes Erkjendelse er Afholdenhedens <b>Moder</b> . Den christelige Klogstab har stedse for Oie <b>Maalet</b> , Klenodiet og retter sit Lob og Kamp efter at gribe det . . . Men hver den , forn kjemper , er afholdende i Alt " ( 1 Kor . 9 , 25. ) . For at det stal lykkes os , forn <b>Kundskaben</b> raader os , maa Afholdenhed vere vor Ledfagerste. Ligner Dyden et godt Trees Stamme , faa maa man sammenligne <b>Kundskaben</b> med Knoppen , som bestandig vender sig mod Lyset og giver ethvert Skud Retningen opad , men Afholdenheden med den beskyttende Bark , der indhyller Treet og alle Grene , eller Kalken , som sammenholder Blomsten , at den ikke falder af for Tiden . . . Holder Eder fra kjedelige Lyster , som stride mod <b>Sjelen</b> , " saaledes formaner Paulus de Fremmede og Udlendinge ( 1 Br . 2 , 11. ) , og tilraaber dem : ..Verer edrue og aarvaagne til Bonnen ! " ( 1 Br . 4 , 7. ) , og ligesom han tilforn har fagt , at de have nndflyet Verdens forgjengelige Lyst , saaledes stulle de numedTapperhed og Viisdom holde og verge sig , at Lysten ikke atter tager dem tilfange . Den handler vifelig , der lever efter det Ord : . . Gak ikke efter dine Begjerligheder , og hold dig fra de Ting , forn du haver Lyst til " ( Sir . 18 , 34. ) . I den tydste Bibel sinde vi det samme Ord , som her er gj en givet med Afholdenhed , ellers overfat med Kydskhed ( Ap . Gj . 24 , 25 ; Gal . 5 , 22. , fml . ogsaa

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016031548122'target=_>Landstad, M.B., 1873, Kirkesalmebog efter offentlig Foranstaltning</a><h4>4262</h4>Du er alene den , fom giver de Tratte Kraft og formerer den Vanmcegtiges Styrke , o saa giv ogsaa mig Svage Kraft og styrk mig Dag fra Dag herefter at blive from og leve bedre . O Gud , hold dit Ord og dine hellige Sakramenter vedlige hos os evindelig uden falsk <b>Lcerdom</b> og vrang Gudstjeneste, at jeg og andre bodfærdige Syndere altid maa finde Kraft deri til Salighed og en Livslugt til Livet . O Herre , bevar Kongen og Landet med al kirkelig og verdslig Ovrighed fra al Ulykke . Gud give os Fred og Rolighed . Gud unde os Helbred og Sundhed . Gud naa ' digen velsigne vor ( Ager og Eng ) Mad og Drikke , vor Kurv og vore Levninger. O Herre , tcenk i Naade paa alle Syge og dem , som lide Angest og Pine . Giv dem en god Nat , fcnd Lindring og Lcegedom , fend dem Trost og Haab . Hjalp dem til at kunne være glade i Haabet , taalmodige i Trccngfelen og varagtige i Bsnnen , og lad deres Fo ' rlsøningstime , fom bie paa dig , ilte voere langt borte . O Herre , vcer mig arme Synder ( min Hnsbonde , <b>Hustru</b> , Forceldre , Børn ) min Slægt og Venner fjern og ncer og alle Mennesker naadig for Jesu Kristi Skyld ! Vogt os med Hus og Hjem , Gods og Formue fra Ilds-Nod , Vands-Vaade , Tyvehaand og alt andet Ondt , og lad ingen Ulykke komme inden Dore . Der lagges Dage , Uger og Aar til min Alder , Vor Klokke slaar , vor Tid bortgaar — og Evigheden forestaar . O Herre , lcer mig saa at talle mine Dage , at jeg maa bekomme <b>Visdom</b> i Hjertet ! Lcer mig saaledes at bruge de dyre Naade- Timer , at jeg altid med Frimodighed kan bie efter min Frafkillelfes Stnnd og forvente i Haabet Guds-B « rns herlige Frihed . Al , Herre Jesu , kom , kom naadig , kom ien god Stund ! I din Haand befaler jeg baade Legem og <b>Aand</b> , du forloste mig , Herre , du trofaste Gud . Amen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012052204036'target=_>Bang, A. Chr., 1874, Hans Nielsen Hauge og hans Samtid</a><h4>844</h4>omtrent paa samme Maade som Nutidens „ Bibelkvinder " ; men dette kunde blot lceres dem ved den historiske Udvikling . Til Bekrceftelse paa , hvad her er sagt , stal jeg anfsre Vidnesbyrd fra Samtiden . John Haugvaldstad strider : „ Et Aar senere kom en <b>Pige</b> , fsdt i Gudbrandsdalen , som og havde Opbyggelse hos os og flere Steder . Ved hende fik jeg megen Ledelse i <b>aandelig</b> Henseende " ' ) . I mine Samlinger har jeg Optegnelser af en gammel <b>Kone</b> , der striver Folgende om sine Forceldre : ~ Min <b>Moder</b> havde i lcengere Tid forud , ja fra sin tidligste Barndom , havt et kraftigt Kald paa sig ; da saa Rygtet kom ud blandt Folk , at En ved Navn Hans Nielsen Hauge reiste omkring i Landet og talede til Folket , de skulde omvende sig , erholde Syndernes Forladelse , tro og blive salig , blev det deres ( a : Foreldrenes ) alvorlige Tanke ; men paa hvad Maade dette skulde ste , forstod de ikke , iscrr min Fader . — Der fandtes da ikke et Menneske blandt dem , som kunde veilede dem . De lcrngtes da meget ester , at Nogen skulde komme og hjelpe dem tilrette . Herren saa deres Lcengsel og tenkte paa dem med sin Naade . Der kom tvende unge <b>Piger</b> , som nylig vare opvakte og forte et kristeligt Liv , disse vare opvakte ved Hauges Virksomhed; dette blev dem ( 9 : Forceldrene ) til stor Velsignelse . " Hauge fortceller , at da hans fsrste Skrifter „ kom for Lyfet " , da var der Flere , der „ bekom Guds <b>Visdom</b> og Kraft til at bekjende Jesu Navn som en Frelser fra al Synd " " ) Saaledes var det ogsaa . Allerede ved Udgivelsen af andet Oplag af Bogen om „ Gnds <b>Visdom</b> " kunde han tale om „ tro Hyrder " , som ei er „ Leiesvende " , saadanne , der „ ei for Penge myrder , eller bruger verdslig Magt " " ) . I Begyndelfen af 1797 trceffer man hans Medarbeidere i og langs Veien til i 1798 optrede de paa Kongsberg og Eker ^ ) ; i 1799 <b>moder</b> man dem paa Modum og i Hougsund ' ) , i Gudbrandsdalen

<h4>2658</h4>det fandt man sig dog i paa Grund af hans Reises Vesiaffenhed og de ovrige Omstcrndigheder ; derimod tog man ligetil Anstod af hans „ udvidede Begreb " og hans mere lette Opfatning af flere Ting , at han f . Ex . barberede sig om Ssndagen , at han endog reiste paa Herrens Dag , at han fyslede med fin Strikning og slig „ Forfamgelighed " ' ) . Uagtet Alt dette var udvortes Ting , var det dog klart , at der ud fra det gamle Herrens Redstab traadte Vennerne imode en fremmed Nand , delte var tydelig mcerkbart gjennem den „ friere Vandel i det Udvortes " . Nogen Indflydelse paa hans indre Liv havde maasie Samvceret med Vennerne under denne Reise ; jeg siger maaste ; thi hans Skrivelser fra Hosten 1809 udviser . omtrent den samme aandelige Forfatning som for , kanste dog med en Smule mere Inderlighed " ) . Brevene i Tidsrummet fra December 1809 til December 1810 derimod udvife paa en glcedelig Maade , hvorledes Hauge efterhaanden fandt tilbage til de gamle gode Stier . Saavidt man kan fe , var det ifcer gjennem 3 Omstoendigheder , at Gud drog ham . I Vegyndelsen af 1810 fik han Efterretning om sin Svoger Kjobmand Loofes Dod , og Tabet af den trofaste Ven og Medarbeider gik ham dvbt til Hjerte , saa han underliden af Graad ikke kunde tale . Hans Broder Mikkel med <b>Hustru</b> og Flere bessgte ham i denne Tid ; de vare sammen paa Teien og trostede hverandre med Guds Ords Trost Der spores et kjendeligt Opsving i hans aandelige Liv fra denne Tid , og Minderne fra Vcekkelfestiden blive mer og mer levende . Saaledes hedder det f . „ Salig var den Stund , Gud kaldte os ved sit Ord og <b>Aand</b> , og lyksalig er al den Tid , vi har med " fri Villie anvendt alle de af Gud betroede Midler til vore og Medmennefters Sjeles Foredling, eller givet det Gode Rum til at virke i vort Hjerte , og sogt at virke paa Andre igjen . Bliver vi nu i den sande Tro og Kjoerlighed med Retfcerd , som i <b>Visdom</b> over Godt , jo mere jo bedre , indtil vor Ende , ak hvor hoi bliver da vor Lyksalighed".

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009072703030'target=_>Besser, W.F., 1867, Lukas's Evangelium</a><h4>74</h4>V . 4. Paa det du kan loere at kjende den Lcerdoms Vished , i hvilken du er bleven mundtlig underviist. Den <b>Lcerdom</b> , i hvilken Theophilus var underviist, er den famme , i hvilken ogfaa vi fra Barndommen af ere underviste af vor <b>Moder</b> , Kirken , saafremt den redelig har forvaltet mod os sit moderlige Embede . Jeg er en elendig , fordømt Synder og bliver uden Fortjeneste retfcerdiggjort ved den Forløsning , som er steet ved lesum Christum , hvis egen jeg er ; dette er Lcerdommen, den saliggjorende Christenlcerdom , og denne <b>Lcerdom</b> har sikker Grund , som staaer übevegelig , naar Jord og Himmel forgaae . Hvad min Herre Christus har gjort for mig , det kan Ingen gjore u ske et , ikke jeg selv , ikke Verden , ikke Djcevelen .  Et Udet Ord kan ham fcelde , " — nemlig dette : F u l d b r a g t . Naar min Sjcels , Fiende vandt saadan Magt over mig , at jeg maatte give Slip paa dette Ord , da var jeg forloren ; men saa lcenge jeg holder fast ved det og stetter mig til , at min Guds og Frelsers Gjerninger visselig og sandelig ere stete til min Forlosning , saa lcenge har jeg Trost og Hjcelp imod al Anfegtelse . Dette er Forskjellen mellem Christenlcerdommen og alle de gamle og nye Hedningernes Lcerdomme : de Christne troe , hvad deres Gud har gjort og enonu gjsr for dem , Hedningerne indbilde sig , hvad deres Guder ikke have gjort og aldrig gjore . — Men saa <b>vis</b> den saliggjorende Lcerdoms Grund end er , den bliver dog übeviist for den , som ikke kan fordrage den , som intet Hjerte har

<h4>1911</h4>sig kjcerlig til denne svage Begynder i Troen og vil ikke , at Disciplene stulle forkaste ham ; havoe Guds <b>Aand</b> bragt det saa vidt med ham , at han ikke var Christi Sag imod , saa var allerede det fsrste og vigtigste Skridt gjort , han var allerede for de Troende , omenstjsnt han endnu ikke havde Tro nok til offentlig og ganjke at slutte sig til dem . Anderledes dommer Herren om dem , der ville dele sig imellem ham og Verden , vel begjoere Trosten af hans Navn , men slye Skjcendselen , der er forbunden med hans Navns Bekj endelse ; angaaende disse se Cap . 11 , 23. , hvor Jesus vender Ordene om og siger :  Hvo , der ikke er med mig , er imod mig . " — Vi burde vel vide , hvad dette Ord : ham det ikke osv. " vil sige os . Forbyder ikke denne Mand , denne <b>Kvinde</b> at fryde sig i Evangeliet ogiLsnkammeret at paakalde Jesu Navn , fordi de endnu ikke have sluttet sig til Eders Menighed , endnu ikke besidde Klarhed eller Kraft nok til i Gjerningen at <b>vise</b> Sandheden A3re i alle Stykker . Er det Eders Bnste og Begjcer , at de skulle bekjende sig til Christum og ikke til Eders Personer , da stulle I ophjcelpe dem som Skrobelige ( 1 Thess . 5 , 14. ) , ret flittig boere dem frem for Naadens Throne , at de kunne voxe i Naaden og fatte Mod til ogsaa udvortes og offentlig at vcere for Eder , det er for Christum iblandt Eder ; thi det gaacr visselig ikke an at bekjende Hovedet og ville fornegte Lemmerne . V . 51. Men det begav fig , da de Dage fuldkommedes, at han skulde optages , da vendte han ftadelig fit Ansigt at vandre til Jerusalem . For sidste Gang forlod nu Jesus Galilcea . Han styrkede sit Ansigt til denne Reise ( det var jo herligen bleven styrket paa det hellige Bjerg ) ; thi , fsr han naaede frem til <b>Maalet</b> ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011051320021'target=_>Rosenius, C.O., 1880, Fra Døden til Livet</a><h4>57</h4>og fortabt , men ved Faderens Kjcerlighed blev oversst med en aldeles uforsiyldt og uforventet Naade . fit ogsaa derved en Saltgheo og et Hicrtc for Faderen , som den anden Son aldrig kjcndtc — et r Mg MM og Kjcerliqhed smettet og brcendende Hjerte , mcdens dcrimod den Son som aldrig var vegen fra Guds Bud og aldrig var fortabt , havde et koldt og forvcent Hjerte imod fin Fader . — Hovcosagen er : Gud vil alene vcere Konge og Herre , alene stor over alle sine Hcenders Gjernmger . De Enqle . som ikke beholdt deres Fyrstendsmme men , strcebendc efter Guds Hoihed og Mre , overgav deres Bong , dem forvarede han med evige Lcenker tll den store Dags Hom ( Ind . 6 ) ; dette var et Fald , for hvilket ingen Hjcclp , ingen Oprettelse fandtes . Mennesket derimod blev ved sit Fald bcstandrg Na adetlgger paa lorden og Gud bestandig stor i Naade og Wre . — Men ogsaa det vi ikte forståa , blivcr lige guddommeligt og stort ; det tllhorer Herrens Mre at vcere übegribelig for os . Derfor gaa vi nu over ill selve den hclliqe Historie og betragte den betydningsfulde Prove og Fald . Og Gud Herren tog Mennesket og satte det t Edens Have ttl at dyrke den og bevare den . Og Gud Herren bod Mennesket og sagde : Du maa frit cede af alle Trceerne t Haven , men af Kundskabens Troe paa Godt og Ondt as det skal Du ikke cede ; thi paa hvilken Dag Du oeder afdet , skalDndoDodcn . " , Dette er nu Provcn , som Adam paa sin Slcegts Vegne Moe beståa. Dette er den Pagt eller det Forbund , som Gud oprettede med Mennesket . Paa den cne Side var den treenige , uforanderlige , majestetiske Gud som den hoiestc Lovgiver , paa den anden Side var Adam , som ! sin Person forestillede hele den mennestelige Slcegt , hvilket Udqanaen viser , da hans Fald og dets Frugter strakte sig til alle Mennester'(Rom. 5 , 12 ; 1 Kor . 15 , 21. 22 ) . Det , som nu her forst bor mcerkes , og som er i hoi Grad vmtigt og lcererigt , er den Omstcendighed . at Gud knyttede sit Forbud til en M ringe og übetydelig Gjerning . Se og leer her engang for alle . hvorledes den store , hellige Gud dommer ! Naar han vilde scette Mennestet paa den storste og mest betydningsfulde Prove , da udvalgte han den allerringcste Gjerning , som kunde ovtcenkes paa lorden . En Prove , som skulde ndstrcekte sine Folgcr over en hel Verden , et Bud , hvis Overtrcedelsc vilde medfore al Slags Dod , legcmlig , <b>aandelig</b> og evig , knyttedes til en saa ringe Gjerning som at cede Frugtcn af et Trcemidt Mandt Paradisets Rigdom paa Trcecr og Frngtcr . Visselig har Herren med dette forfte Bud for cvindelig Tid givet Mcnnestcslcegten den <b>Lcerdom</b> , athansNinesc ester Hjertet , efter Lydigheden , ikke efter Gjerningens Stor relse . Vissclig har han givet os en alvorlig <b>Lcerdom</b> om , at han vil vcere en altraadcnde Herre over sine skabte Vcesener , som fordrer Lydighcd , fuldkoinmcn Lydighcd , blot Lydighed , naar han befaler noget . Havde han f . Ex . sagt til Adam : Du stalikke drcebe din <b>Hustru</b> , eller : Du skal ikke pine Tiyrcnc — da havde strax Fornuften seet Grunde dertil , billiget og understottet Vudct . og saa rette fig derefter blot ' , for det Fornuftige i Budet : men da havde Lydigheden — hvad vi kunne kalde den rene Lydighed mod Herrens Bud — aldrig rommet paa Prove . Da havde Mennesket strax heftet sig ved Gjerningernes Vcerd og yelt overseet , hvad Gud cgentlig vilde have ,

<h4>71</h4>og jom har efterladt saa dybe Mcerker hos alle Adams Born , at de endnu den Daa idag danne den egentlige Grundkilde til al <b>aandelig</b> Fordcrrvelse , alle de mcegtigste Fnitelser og de dybeste Fald , det er Slyngens Indskydelser til Hovmod , Selvforgudelse og Uafhcengighed. Allerede i Slangens forste Ord : ~ Har Gud virkelig sagt : I stulle ikte crde af alle Trcrer i Haven ? " ligger den Tanke skjult : I , ' - I , Guds Bom , I . Guds Herligheds Vcerk - I , Herstere over den hele Jord — skulde I have et Baand paa Eder ? skulde ikte I have fuld Frihed ? Men siden , naar Fristerens Dristighed er modnet , da siger han ligefrem : ~ I skulle blive ligesom Gud . eller nsiagtigere : ~ Guder " . Og just dette : ~ I stulle blive ligesom Gud , " — ikke ved nogen Guds Naade og Gave , men ved Eders egen ~ Kundstab om Godt og Ondt " . ved Eders egen Forholdsregel , egen Gjerning , det trcengte sig dybt ind t Mennesket og efterlod sig hos Adams Slcegt Mcrrker . som end den Dag , dag uophorlig stlkke frem . . . Og Kvmden saa , at Hr ceet var godt at crde af , og at det var en Lyst for Vmene , og at Trceet var onsteligt til at faa Forstand af , og hun tog af dets Frugt og aad , og hun gav ogsaa sin Mand med sig og han aao . " Her er det stillet sammen Alt , hvad der bevcegede <b>Kvinden</b> til at jynde . For det Forste var Trceet godt at cede af og lysteligt at se til Der er selve Begjcerligheden eller Syndens Behagelighed For det Andet var Trceet nyttigt , efterdi det gav Forstand . Der er nu Frugten af Dicevelens Vrcedikcn . Det var saa langt fra , at hun troede Guds Trujel eller frygteoe noacn Straf , at hun tvertimod ansaa det nyttigt at cede af den forbudne Frugt . Dcrhen kan Djcrvelen dreie Sågen Og her har yl nu de tyende Hovedstykker . hvormed han i Fristelsens Stund altid udjmykker Synocn , ncmlig idet han for det Forste fremstiller den ion , jaarc behagelig for Kjodet og for det Andet udsmykker den som aldeles ikte farlig , tvertimod nyttig . I den samme stjonne Skikkelse lyste Synden ogsaa for Judas , da Djcrvelen havoe givet ham den Indskydelse. at forraade Jesus . Det maa uofthorlig have hvistet i hans Indre : . . Tcrnk , tredive Solvpenge ! tcenk , hvormegen Nytte og Nydelse du kan staffe dig for den Sum . Desuden er der jo slet lngen Fare forbunden dermed i Jesus er uskyldig , og vi have Lov og Ret ; han kan ikke blive dsmt . Men hvorom altmg er , saa tcrnk : tredive Solvpenge ! Saaledes gik det ogsaa her med Eva . Hun saa paa Trceet . saa paa den forbudne Frugt , saa paa Fristelsens <b>Maal</b> . det var lysteligt . og hun Horte paa Slangen , som sagde : ~ I stulle visselig ikke do . " Og Trceet bliver hende mere og mere lysteligt , jo lcengere hun ser derpaa og horer Slangen . Her gildet , som Apostelen siger : ~ Naarßeajcerligheden harundfanget , fsder den Synd , men naar Synden er fuldkommen , fsder deu Dsd " Og saa ~ tog hun af dets Frugt og aad ; og hun gav ogsaa sin Mand med sig . og han aad . " O , forfcerdelige Stund ! Eva tog — tog den Frugt , som den almcegtige , den store , gode Gud havde forbudt hende at tåge — hun tog og hnn aad — og hun gav ogsaa sin Mand deraf — og han aad . Nu var det gjort . Og hvad fulgte saa derpaa ? — Lader os hore , hvad Ordet siger derom : . . Da oplodes BeggesOine , og de forstode , at de vare nogne ; og de heftede Figenblade sammen og gjordesig

<h4>124</h4>seas hedcr det : . . Icg vil forlose dem fraHelvedes Vold , jea vll gienlsse dem fra Dodcn ; Dod ! jeg vil vcere din Pestilentse ; Helvede ! ieg vil vcere din Ddclceggelse . " Hos Micha heder det : . . Og du , Bethlehem Ephrata , er du liden at vl-erc blandt Indce Tusinder ? af mg skal mig En udgaa til at ucere en Herster i Israel , og hans Udgange er fra fordum fra Evighcds Dage " . Hos Haggai hcdcr det : Thl saa sagde den Herre Zebaotl ) : Endnn engang . om en liden Stund , saa vtl jeg bevoege Hunlene og I orden , og Havet og det Torre , og alle Hednmgernes Lcrngsel stal tomme " , . vos Sacharms heder det : . . Thl se , leg vil lade mm Tjenee Zemach ( Vext . Spire ) komme . Thl se . der er den Sten ... paa den cne Sten er syv Nine ; se . jeg vil udgarve dcnsUdgravning. siger den Herre Zebaoth . og jeg vil borttage dette Folks Misgjerning paa ccn Dag " . Hos Malachms heder det : Se , jeg sender mm Engel , at han skal berede Veicn for mit Ansigt ; og strax skal den Herre komme tll sit Tempel , hvilkcn I opsoge . og Pagtens Engel , som I have Lyst til at se , halt kommer , " Fremdeles heder det hos Esajas : . Men han , han er saarct for vore Ovcrtrcedelser . er knust for vore Mtsmerninger ; Straften laa ftaa ham . at vi skulde nv.de Fred og vi have saaet Lcegedom ved hans Såar " . Enduidere hos Mlcha ; . . Han stal Wen forbarme sig over os . han stal undertrykte vore Msgjerninger ; za du skal kaste alle deres Synder i Hauets Dybhcder . " Fremdeles hos Sacharias: . . O du . ved dm Pagtes Vlod udlod jeg dine Bundne af en Huie , hvorder ikke var Vand ndi " og endelig : . . Pau den samme Dag stal vcere en aabnct Kilde for Davids Hus og for Icrusalems Indbyggere lmod Synd og imod Urcnhed . " ( Esai . 53. ler . 31. Ezech . 37. 25. Dan 9. Hos . 13. ' 14 Mich . 5. 2.3 ; 7 , 19. Hag . 2. 7. 8. Sach . 3. 8. 9 ; 9. 11 ; 13,1 . Mal . 3.1 ) . Flere Vldncsbyrd behoves ilke — se . hvllken lysende Rad af Vldner! Saaledes have de Alle . som med en Mund , ved Herrens <b>Aand</b> Mdnct om den kommende Frelser . Helten . Slangeknuseren — og lcengtes og sukket : . . Hvo vil give Israel Frelse af Zion " ( Salme 14.7 ) ; . . gw du vllde sonderrive himlene ! gid du vilde fare ned , at Bjergene kunde bortflyde for dit Ansigt . " ( Esai . 63 , 19. ) Saa vidncde de hcllige Mcend ; saa sukkede de Fromme af Loengsel ester Zions Trost , Men han . som raader for Alt , havde efter sm Vlljes Behag sclv bestemt det Tidspunkt , da Verdens Middag skulde frembryde . og Solen med fin Glands forjage Morgenens tykke , morke Taaaer . ~ Der Tidcns Fylde kom . ulendte Gud sin Son . fodt af en Kvmde . fsdt under Loven . paa det at han skulde frikiobe dem . som vare under Loven ; at vi sinlde faa den sonligc Udkaarclse " ( Gal . 4. 4 ) . « Ogidet han indtrcedcr i Vcrdcn siger han ( til Faderen ) : Slagtoffer og Madoffcr vilde du ikte , men et Lcgcmc beredte du mig . Se jeg kommer — der strevet om mig i Vogens Nollc . — forat gjsre . Gud ! dm Vilje " . Saa staar der strevet i Vrcvet til de Ebrceer 10 , 5. 7. ) Itke Slaatoffer var det , du vilde have . o Fader ! men mit Leaeme hengivet i Doden tll Syndcrnes Forladclse , — <b>Kuindcns</b> Sced . et sandt Menneske gwet som Offer for alle Menncsker . . . Et Legeme beredte du mig " efter Skrifterne . thi . . der er skrevet om mig " efter Skriftcrne , thi der er strevet om mlg r Bogens Nolle " . O hvilkct . . Rigdoms Dyd baade paa Gnds <b>Visdom</b> og Kunoskab ! hvor urandsagelige cre hans Domme og hans Veie usporhvcrkcn

<h4>126</h4>lige ! " » Thi den , som ikke vidste af Synd , har han gjort til Synd fer os ( moerk ! mcrrk ! ) , paa det vi stulle i ham vorde retfoerdige for Gud " ( 2 Kor . 5 , 21 ) . ~ Det . som var Loven umuliat . idet den var lrafteslss formedelst Kjsdct , det gjorde Gud , da han senote sin egen Ssn i syndigt Kjods Lignelse og til et Syndoffer og straffede Synden i Kjsdet " ( Rom . 8 , 3 ) . har Gud elstet Verden ! " — Men hvo kan ovregne alle disse trostelige Ord ? Gud give os <b>Visdommens</b> og Aabenbarelsens <b>Aand</b> til l Saudhed at kjende ham ! Han ovlyse vor Forstands Oine til at forståa det Haali , hvortil vt ere kaldte , og begribe med alle de Hellige . hvad der er dci Brcoc oa Lange og Dybe og Hoie og kjende Kristi KjcerliaHed , som ovcrgaar Kunostaben . Ai have hort Gud selv vidne om sin Sm . at hau som den <b>Kvindens</b> Sced skulde sone for Faldet . De helllge Profeter, Engle og hele Evangelisterncs store Stare bevidne det samme . Kristus selv , han som er . . det tro Vidne " har jo kundgjort , at han var kommen for at ~ give sit Liv til en Gjenlosningsbetaling " og ~ slt Blod ttl Syndernes Forladelse . " Hvem i Himmelen eller paa lorden siulde jeg da frygte og cere uden den ene sande Gud , Himmelens og lordens Skaber , som saaledes selv ester sin egen fri Viljes Behag besluttede vor Frelse og Forlosning ved sin enbaarne Ssn ? ! Er ikte dette en uomstsdelia Sandhed , <b>vis</b> og fast i al Evighed , saa gives der intet Vitt og Fast i Hlmmelcn eller paa lorden . Om Syndefaldets Virkelighed har jeg dagligcn Viduesbyrd i alle Lemmer ; om Lovens . Dommens og Truslernes Virkelighed og Sandhed er Samvittigheden mig et uimoosigeligt Viduc ; men er alt dette vist og sandt , saa er det ligesaa vist oa stndt , hvad der staar strevet om vor Frelse i Kristo . At hele Verden saldt og blev fortabt ved Adams Fald . det er sandt ; at Doden trcengte ajennem til alle Mennester , det er oasaa sandt ; at den strenge , helliae Lov for < domte os alle ial Evighed . oet er oasaa sandt ; at yl alle . alle ere Syndere, urene , forgiftede . sulde af al Ondsiab . det er ogsaa sandt ; at Hjer « tet er bedrageligt mere end alle Ting . letsindigt , hykkelsk , haardt og koldt , det er sandt ; men at Gud i Kristo udvalgte os fsr Verdens Grundvold blev lagt . det er oasaa sandt ; at Kviudens Soed . Slangeknuseren . virke » lig har afstaffet DMelens Gjerninger , det er ogsaa sandt ; at han blev gjort til Synd for os . paa oet at vi stulle i ham vorde retfcerdige for Gud . det er oasaa sandt Gud var i Kristo og forligte Verden med sig selv , det er ogsaa sandt- at vi nu saaledes alle have i Kristo faaetNaade over Naade . oet er og bliver — Gno verre evig lovet oa priset ! — ogsaa vist oa sandt . Derfor bekjende vi . leg tror . at Jesus Kristus har gjcnlost mig , som var fortabt og fordomt til den levige Dod og har fnet mia fra Synd . fra Dod og fra Satans Rige — kort : han har senderknuset Slangens Hoved . Men lader os nu trcede lidt ncrrmere og se . hvorledes det glk tll l denne Strid og med denne Seiervinding . Vi ermdre jo . hvorledes Stillingen var . Guds Lov var hellig og uforanderlig ) om Gud selv ; den kunde ikke rykkes ; ikke et eneste Bogstav . ikte en Tsddel stulde forandrcs Paa den anden Side var Mennesket helt lmennem forgiftet , jyldt af Slanacns Sced ; her var altsaa en evig Dod uundgaaellg . Dummcn lod ju : Dm , der synder , stal ds. Men der nu formedelst vor Guds indcrlige Bariuhjcrtiahed den velsignede Kvmdesced kom for at hjcelpe og frelst sin arme faldne Slcrgt . saa begyndte han med at blwe

<h4>256</h4>blive varme af Kjcerlighed ogGudsfrygt ; thi Gud er en saa stor og brcendende Kjcerlighed , at ingen kan kjende ham , uden at blive op « tcrndt af ham til Gjenkjcerlighed , og Kjcerlighed er jo <b>Moder</b> til alle gode Gjerninger , ~ Kjcerlighed er Lovens Fylde . " I Korthed : de , som mene sig at vrde alt , hvad der vedrprer Evangelium, de have endnu ei lcert de forste Vogstaver deraf . Apostelen siger : Dersom nogen tykkes sig at vide noget , han haver endnu aldrig vidst noget saaledes , som det bor sig at vides ( 1 Cor . 8,2 ) . Den Troende tan vel i Anfegtelsens Mprke i den Grad omdrives og seres vild , at han glemmer , hvad hans egen Erfaring har lcert ham , saa at han atter begynder at spgeiLoven , hvad aldrig er der at sinde ; men han forbliv er ikke i en wadan Vildfarelse , han lader sig gjerne sige : slatter altid Evangelium heit og forbliver altid et Diebarn ved dets ~ Trpsts Bryst " ( Ej . 66,11 ) . Apostelen sagde : Dersom nogen tykkes sig at vide noget , han haver endnu aldrig vidst noget saaledes , som det sig bpr at vides . Dette har i Scerdeleshed sin Anvendelse paa Ovangeliet. Evangeliet indeholder Ting af den Beskaffenhed , at den , som ser , hvad det indeholder , ikke formaar at tro det , medens den , som har let for at tro det , visstlig ikke ser , hvad det har til sitlndhold . Vel tykkes han at se , hvad det indeholder , og det saa bestemt , at han kunde svcerge tusinde Eder dervaa , men hans hele Vcesen benegter det . Luther siger sandt , naar han siger : at den , som kunde ret baade fatte og tro , hvad Evangelium indeholder , ~ han kunde ikke lcengere leve her paa lorden , men maatte op af overvettes Glcede . " Sandelig , ikke skulde en saadan vcere saa kold , stiv og uden <b>Aand</b> , som disse udlcerte og mcette Aander ere ; ikke skulde det falde saa tungt at efterfplge Kristum , at elske , tjene og bekjende Krittum , at bede og foriage m . m . , hvis man ret troede , hvad Evangelium siger os . Men se her er en Gruno , hvorfor mange , som altid lcere , dog aldrig tomme til SandhedsKundstab ! Jesus sagde : Ingen kjender Spnnen uden Faderen , og Ingentjender Faderen uden Smnen og den , som Spnnen vil det aabenbare ( Matth . 11 , 27 ) . Naar ikke Sonnen vil aabenbare det for et Menneske , saa er alt forgjceves for det . Kort fpr ( V . 25 ) havde Kristus sagt , for hvem han ikke vild e aabenbare det . : — ~ Du haver fkjult det for de <b>vise</b> og forstandige " — d . e . saadanne , som i egne Aine ere tundskabsrige og mene blot ved Tcenkning og Studering at stulle kunne tilegne sig Evangelium . Man lcefer og hprer Evangelium paa samme Maade som en verdslig Videnskab , uden derved at bsie sit Hjerte for Gud ; for saadanne har han behaget at skjule det . la , Fader, thi det var saaledes behageligt for dig ( V . 26 ) . Paulus var vel en Mester i tyoeligen og rimeligen at forklare ogudlcegge Evangelium , men ingenlunde anfaa han det for not for Menneskene , at de blot hsrte hans Prcrdikener og lceste hans Breve ; tvertimod , han ansaa alligevel alt saa ganske at bero paa Guds ~ Aabenbarelse " , Guds Meddelelfe af ~ VisdommenK og Aabenbarelsens <b>Aand</b> " , at han idelig anraaber Gud derom for sine Menigheder . Alene i Brevet ttl de Efeser omtaler han tyende Gange , hvorledes han paa sine Knee raabte og bad til Gud for dem , ~ at vorcherres Jesu Kristi Gud , Herlighedens Fader , vilde give Visdoms og Aabenbarelses <b>Aand</b> udi hans <b>Kundskab</b> , eders Forstands oplyfte j2ine ' , at I kunne kjende , hvilket det Haav er , som han kaldte eder

<h4>560</h4>Kristne , som ere vaagne og hender sin Synd . Der findes ingen , som jamrer og beklager sig saamegct over Synden som de Helliqe . ledes gik oet da , om Gud skulde domme os efter sin Lov ? Maa itke de hellige bede : ~ Gaa ikte til Doms med os : thi for dig er ingen levende retfcerdig ! " Saaledes vidner Skriften : ~ lblandt hans hellige er ingen ustraffelig . " ~ tzvad er et Menneske , at det skulde vcere rent , eller at den skulde vcere retfcerdig , som er fodt af en <b>Kvinde</b> ? Kimlene ere ikke rene for hans Dine ; hvormcget mere er Mennesket veocrstyagellgt og stinkende for ham og drikker Uretfcerdighed som Vand " ( lob . 15 ) . Og atter siger Skriften : ~ Der er ingen retfcerdig , end ikke En . Alle ere afvegne , de ere tilsammen blevne uduelige , der er ingen , som gjor godt , der er end ikke En " ( Rom . 3 ) . Hver hellig behover daglig Syui ) crncsForladelse . tzvorledes vilde det da gaa os , dersom vi enonu siuldc dommcs efter Loven . Det er meget vigtigt , at vi af Ordet ret lcere at kjende og tro vort dybe Syndefald og Forocervelsen i al Menneskenatur , som ogM endnu er tilovcrs hos de Hellige i deres Kjod , ei alcne for at vi maatte vel betcenke , vurdere og lovprise den uudsigelige Naade , at vi ere fri fra Lovens Domme , men ogsaa for at Samvittigheden maa kunne forbli v e i denne Frihed . Thi om vi ikke af Ordet ere vel underrettcde om denne Fordcervelse , saa skulle vi med al <b>Kundskab</b> om Kristi tjcncstc dog aldeles nedsylcke i Elcndighcd og fortvivle , saasnart vi erfare den hos os selv . Thi om jeg , som tror paa Kristus , finder en Syndeelendighed hos mig , som jeg ellers tror , kun alene kan findes hos de Vantro yg aldrig hos Guds Born , saa hjcelver al Kristi Fortjcneste mig ikke / thi det hcder da : ~ Du har hverken Del eller Lod i dette Ord- thi dit Hjerte er ikke ret for Gud " ( Apgj . 8 , 21 ) . Derfor maa vi endnn noget noiere overveie dette Punkt . Vi saa i Skriftens Vidncsbyrd , at ogsaa . . iblandt Guds hellige er ingen ustraffelig , " at ~ dcv er ikte til , som er rctfcerdig egen Person), ikke En ; " men saa tccnkcr ofte en cnfoldig troendc som scedvanlig blivcr anfcgtct mcd alle Modsigclser af Hjertet og Satan : Ia ingen ' er ustraffelig , ingen er rctfcerdig ' for Guds Oine , for hvem ikke cngang Himlcne er rene , Guds Oiiie , som se det , der ikke er , men aldrig kunne de sande Hellige have og fole saadan virkelig Synd i sine Hjerter , som f . Ex . syndige Lyster ' og Begjceringer . Koldsindighed og Mistro mod Gud , tzovmodstaukcr o . dcsl , ; thi Helliggjorelse , den Hclligaands Verk , ja Navnct ~ hellige " maa ogsaa betyde noget . Naar jeg nu soler saadant hos mig , som er virkelig Synd , ja det udbrydcr i Ord og Gjerningcr , saa kan jeg jo umulig have nogen Del og Lod i Kristus og hans Fortjencste . " Og paa denne Maadc ster det , blot ved Misforstaaelse af de Helliges Hellighcd og ved Myndighed om den almindelige Fordcervclse . at de som virtelig kjende sin Synd , de gaa med alt det Evangelium , de hore , dog i en hcmmelig Selvfordommelse og savnc den Tro og Fortrostning , som skulde give oem Kjcerlighcd, Liv , Lyst og Kraft til ' det Gode , og kun de troste sig . som ere sovende vok til ikke ' at mcerke Syndcn hos sig . Se . det er sormedelst dcunc Tankeforvirring , det blivcr saa nodvenoigt ogsaa at betragte saadannc Steder i Skriften , som med levende Exemftler <b>vise</b> os , hvao der menes dermed , at ~ iblandt de Hellige er ingen ustrasselig " .

<h4>718</h4>hadde ogsaa for Kristi Skyld forladt sine forrige Vcrdeuslyster samt alle Kunstgreb og Uredeliaheder i sin Bedrift ; mm de begyndte snart at tabc og savne noget i det lordiste ; de vilde en Tid bie ester Herren , men knndc lkke holde ud ; de begyndte derfor at tcenke : Nei . jeg har <b>Hustru</b> og Bornatforsorge , jeg maa t det Ydre stille mig saaledes . at jeg igjen kan vinde mine furrige Kunder ; eller : Jeg maa tilligc dcgynde en ny Nocringsvci o . s . v. Nu sik den arme Sjcel saamcgct vcrdsligt i Hjertet og Huset , nye Selsiabcr og nye Spekulationcr . at det Aandelige maatte nn hvile . de aandcligcßsger lagdcs tilside . de aandcligc Venner var der nu neppe Tid at tale med — snart var ikte Soudagcn ledig til Sjcrlcns Pleie . Den samme , som for talede om sine Synder med Vckymrmg og om Naaden med Glcede . var nu stum om disse Ting . Kom en redelig Ven ham imodc med den forsigtigste Advarscl . saa taaltes ei dette , men enten asvistc han denne Omsorg med et bidcndc Svar eller ogsaa sorargcdcs han hemmelig Verdcns Agtelse . Vcnsiab og Omgang blev ham beslandlg kjccrcre . Vcrdcns Born vare bchageligcre end de for saa kjcerc aandeilgc Venner , disse synes ham nn saa ~ sucvcrhjertcde og dadeljyge". Saaledes er han nu forandrct . Sindct furveudt . Kjcerligheden tjolnct - den gode Sced kvalt . Og dette begyndte blot med en Strcebcn ester at vinde mere l det Jordiske . Saa forstraMligt sandcs Apostelens Ord . . De , som ville vordc rige . falde i Fristclsc og Snarer og i mange daarlige og siadclmc Vcgjceringcr . som ncdscente 3 ) lcnnestcne l Ndeloeggclse og Fordcervctse , thi Pengcgjcerrighed er en Nod til alt ondt . Da nogles Lyst stod til denne , fore de vild fra Trocn og have gjenncmstnnget sig sclv med mange Smerter " ( 1 Tim . 6 , 9.10 ) . Ia « mange Smerta,"'__ Ogjaa dette er en moerkvcerdig Sandhcd . at de , sum ville vcerc rige , gjore sig mange Smerter allercde i Tidcn og gaa tilsidst ligesom Judas ' med sine Solvpcngc i grccssclig Fortvivlclsc ind i Evia.-hedcn, naar de tilsidst ved Dodcns og Evighedcns aluorsfulde Ankomst se , hvor trolost de have foragtct den trufaste frelser , som havde kaldt dem tll Arvmger as de himmelste Nigdommc , disse have de forspiidt og maa nu iordisic ! Herhidhore ApostclcnsOrd : « Velan nn . I Rige ! arcrdcr og hyler over de Elcndighcdcr . som komme over edcr ! Edcrs Rigdom er raadnct . og cders Klcedcr cre nwlcrdtc ; cdcrs Guld og Solv er forrustct . og deres Rust stal vcerc til Vidncsbyrd imod edcr og crde cdcrs Kjod som en Ild " ( lak . 5 ) . Derfor , hvilken Naade og gu ! ) dommelig <b>Visdom</b> at lade sig noic ogsaa med det Ringe , blot at more sin Kaldsgjcrning med Flid for Herrens Skyld og siden tåge . hvad han gwer ! Saadanne VNcnnestcr ere de lykkcligstc paa lorden . I 1 Tim . 6 staar : ~ Naar vi have Fode og Klccdcr end tavvclige ) . saa siulle vi lade os nsie ; thi Gudsfrygt med Noisomhed er Vinding not " ( V 6. 8 ) . Og den <b>vise</b> Konge siger : . . En trufast Mand bllvcr megel vclsignet ; men den , som tragter ester at blive rig . stal ikte blive uskyldig " ( Ordsft , 20. 80 ) . Fattigdom og Bekymrinaer ere mcegtige Torne til at kvcrle den gode Sced . men aUermcegtigst blivcr det . naar det hastig vender sig saa . at Rigdom og Vellyst folge paa Fattigdom . En omvendt Inglmg — han var from og gudfrygtig . saalcenge han enduu ilke havde mere end . som Ordsvroget siger . . . en Klcednina og en Gud ; " men et rigt Giftcrmaal og en indbringende Vcsttlllng fatte ham hastig i en ny Vcrdcn med

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016110729004'target=_>, 1848, Huus-Postil, eller En Samling af forskjellige Forfatteres, meest Luthers, Franckes og Pontoppidans, Betragtninger over Søn- og Fest-Dages deels Høimesse-, deels Aftensangs-Texter, samlede til en Aargang, og udgivne i den Hensigt, at befordre og vedligeholde christelig Huus-Andagt</a><h4>411</h4>3. Men er der en <b>vis</b> Omvendelse , som de allerede Omvendte ikke have behov , og ikke kan passe sig paa dem : saa er der dog en anden Slags Omvendelse, som de aldrig kunne undvære , men daglig trcenge til at ove , saalænge de leve i denne syndige Verden , førstaae her , den daglige Omvendelse , den daglige Fornyelse og Forbedring , den daglige Renselse fra Aandens og Kjødets Besmittelse . Den kaldes Omvendelse , fordi de , som sve den , ikke længere ligge i Synden , og ikke først begynde at reise sig af den , men ved Guds Kraft allerede tilforn ere opreiste med Christo , og vandre nu iet nyt Levnet . Men da der paa denne nye Vandrings Vei <b>moder</b> dem en eller anden Vanskelighed , Anstod , Fristelse , Besmittelse , Snublen o . s . v. , see saa kjende de det med Bekymring , og hjertelig Ulyst , hjertelig Lcengsel efter mere Kraft , Viisdom og Trostab , de tage sig og sandelig for at komme Fuldkommenheds <b>Maal</b> nærmere , de vinde ogsaa , endstjondt med Kamp , nogle Fordele over det Onde ; og denne de Staaendes daglige Omvendelse , er det da jeg siger , at alle , ja absolut alle sande Christne have behov . Hvorfor ? fordi , at dersom efter St . lohannis Ord nogen siger , han haver ikke Synd , ikke overblevne Fiender at stride imod , ikke Synds Levninger at rense sig fra ,

<h4>585</h4>Deraf kommer det , at mange , nåar de komme i Uenighed med deres Næste , da holde de sig fra Sacramentet , og ei ville bede et Fadervor ; thi de Ord stode dem for Panden , som de bede , nemlig : forlad os vor Skyld , som vi og forlade vore Skyldnere : i Tanke , at saasom de ei forlade , de da stulle afsige Dommen over sig selv , at Gud ei stal forlade dem . Det er og sandt , , man tor eiheller tænke andet ; vil du ei forlade , saa vil Gud eiheller forlade dig . Betænk derfor vel , hvad for et Bad du tillaver dig , nåar du beholder og ei lader Vreden fare . Du gjor ei heller din Sag bedre med det , at du ei beder , og ei vil gaae til Sacramentet , men kun voerre . Thi Gud dommer dig i den Stand , han sinder dig . Følg derfor Christi Raad ; ydmyg dig for Gud , og bekjend dine Synder ; førson dig med din Næste , og lad Vreden fare . Ofte da din Gave : c . ; thi ellers ' horer du , at Gud ei stjotter om dig , og ei vil behage dit Offer og Bon . Det mener Christus , naar han siger : lad din Gave blive paa Alteret . Som han vilde sige : du udretter dog intet dermed hos Gud . Det er det ene Partie , som giver Aarsag til Vrede , og som foruretter sin Næste ; de stal ei alene ydmyge sig for Gud , og bekjende , at de have gjort ilde , men endog for deres Næste , og saa kan de vente Forladelse af Gud . Det andet Partie er , som forurettes , og mener sig at have Aarsag til Vrede . Dem paaminder , og Herren her , at de stal tilgive og ei længe lade sig bede . Thi de ere og , siger Herren , i stor Fare . Thi dersom du længe lader dig bede , og ei er villig til at lade dig førsone , saa vil din Modstander befale Dommeren , Gud i Himmelen , Sagen , og sige : Herre ! jeg haver gjort det , jeg stulde , hos dig , sinder jeg Naade , men ei hos Menneskene ; velan ! jeg vil befale dig Sagen . Dersom nu Gud vilde overfalde dig , hvad mener du da at din Dom blev ? dig , som ei vil tilgive og forglemme , dig vil han tage , og antvorde Tjeneren dig , og du stal kastes i Fcengsel . Der stal du ei komme ud af , forend du betaler den sidste Ekjærv . Det er , du stal ingen Naade sinde , som han siger i Luc . 6,38 . : med den Maade , som I maale med , stal eder igjen imaales . Altsaa vil Herren have af begge Partier , at man stal være barmhjertige , lade Vreden fare , og være venlige mod Enhver . Thi ellers anklager det femte Bud os , at vi ere Manddrabere for Gud , og altsaa vil Straffen ei blive borte . Denne <b>Lærdom</b> og Levnet stal blive blandt de Christne , og flittig oves . Ellers behager det ikke Gud , og vi gjore os selv ei alene Skade paa Legem og Gods , men endog paa <b>Sjelen</b> , hvormed Herren truer . Men udenfor Christi Rige i verdslige Embeder og Rettergang , der haver det femte Bud ei saaledes sin Gang . Der haver Gud givet en scerdeles Befaling , som man stal folge og efterkomme . Thi den , som er i det verdslige Regimente , haver Befaling , at han stal blive vred , straffe og slaae ihjel , naar Undersaaterne begaae noget Übilligt , og det , som er Døden værd . Ligeledes haver Fader og <b>Moder</b> i Husene en scerdeles Befaling af Gud over Born og Tjenestefolk , at de ei stal lee , naar Bornene og Folkene førsee sig ; men de stal sætte dem i Rette og straffe dem . Det befaler Gud dem , og dersom de ei gjore det , saa ere de Gud

<h4>1129</h4>herom , er og strevet os til Advarsel , og omendstjondt Ingen cr , har været , eller maastee torde ventes i vore Tider , og hos os iscer , som Evangelii Lys skinner saa klarligen iblandt , der skulde virkelige « udgive sig for at væreChristus, saa have vi dog virkeligen hsi Aarsag til at vogte os for en falsk Christus. Hvorledes da ? Er det ikke en falsk Christus , naar denne sande Christus som har aabenbaret sig for os i Evangelio , ikke saaledes erkjendes , laeres og troes , som han har aabenbaret sig for os , men man forvender den <b>Laerdom</b> ikke alene om hans Person , og negter hans evige Guddom , og at han er den sande Gud over Alting , velsignet i Evighed , R0m.9 , 5. : c . , men og om hans Midler-Embede , naar man negter hans Forsoning og Fyldestgørelse , som er den eneste Grund til vort Haab , ja til al vor Retfærdighed , Liv , Kraft og Salighed- og at vi stuUe blive retfærdiggjorte uforskyldt af Guds Naade ved den Igjenlosnmg , som er , i Christo Jesu , hvilken Gud haver fremsat til en Naadcstoel formedelst Troen i hans Blod , til at <b>vise</b> sin Retfærdighed for de forgangne Synders Forladelse udi Guds Taalmodighed ; til at <b>vise</b> sin Retfærdighed paa den nærværende Tid , at han maatte være retfcerdig , og rctfcerdiggjøre dem , som ere af Jesu Troe , Rom . 3,24 - 26. Er det ikke en falsk Christus , naar man troer , at Christus er kun kommen til at forlade Synderne , men ikke tillige til at rense os fra al Uretfærdighed , l loh . 1,9 . og at forlsse os sra al Uretviished , og rense sig selv et Eiendomsfolk , som stal være nidkjært til gode Gjerningcr , Tit . 2,14 ; naar man vil bruge Christi Fortjeneste , og den Guds Naade , som os i ham tilbydes , til Uteerlighed , lud . 35. 4 , og til Ondstabs og Hykleries Skjul ? Er det ikke en falst Christus, naar man ikke ligesaavcl vil have Christum til sin Prophet og Laerer , til sin Konge og Regjerer , som til sin Ypperstepræst og Forsoner ; ligesaavcl have ham til Viisdom , Helliggjørelse og Gjenfsdelse , som til sin Retfærdighed? 1 C0r.1 , 20. Er det ikke en falst Christus , naar man negter , forkaster og forhaaner Jesu hellige <b>Laerdom</b> med Ord eller Gjerning ; naar man mener , at man kunde nok være hans Discipel , uden at hade Fader , <b>Moder</b> , <b>Hustru</b> , Born , Vrodre , Sostre og tilmed sit eget Liv , Luc . 14 , 26 , uden at førsage alt det , man har , 35. 53 ? naar man tænker , man kunde nok komme efter Christum , uden at fornegte mig selv , tage sit Kors op daglig og folge ham , Luc . 9 , 23 ; man kunde nok være den Hsiestes Barn , uden at elste sine Fiender : c . Matth . 5 , 44 - 48 ; man kunde nok tjene og behage baade Gud og Mammon, baade Gud og Verden , stjondt Christus siger , det er umuligt , Matth .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009082700002'target=_>Taine, Hippolyte, 1877, engelske Literaturs Historie</a><h4>274</h4>snoe sig hen over det gule Sand , og hvis Landser og Solskjærme danne een Linie og tegne sig som en Fryndse mod Horisontens uplettede Hvidhed ; indiske Paradiser , hvor Roser og atter Roser mylre i titusindvis , hvor Vandspringenes perlende Fjerbuske slynge sig mellem hverandre , hvor Lotosblomsterne udfolde deres brede Blade , hvor tornede Planter reise deres hundrede tusinde Blomsterbægere, medens fromt tilbedte Abekatte og Krokodiller kravle om mellem deres tætte Slyngninger . Imidlertid lægge Dandserinderne Haanden paa Hjertet med dyb og ædel Følelse , Elskerne synge , at de ere nærved at døe , Tyrannerne true med deres Basstemme , og Orkestret raser , det følger de skiftende Stemninger med sine Fløiters smægtende Suk , sine Basuners skræmmende Klang , sine Harpers himmelske Melodier , indtil det tilsidst , i det Øjeblik hvor <b>Heltinden</b> sætterFoden paa Forræderens Hals , bryder ud i Jubel med sine tusinde skingrende Stemmer forenede til en eneste Samklang . Et godt Stykke ! Man gaar bort , blændet og bedøvet , Sandserne svigte under denne Oversvømmelse af pragtfulde Indtryk ; men naar man kommer hjem , spørger man sig selv , hvad man har lært , hvad man har følt , og om man virkelig har følt Noget . Alt i Alt er der neppe andet her end Dekorationer og Iscenesættelse , Følelserne ere kunstlede , det er Operafølelser ; Forfatterne ere kun dygtige Folk , der kunne levere Operatexter og malet Lærred , de have Talent og ikke Geni , de hente deres Ideer ikke fra deres Hjerte men fra deres Hoved . Det er det Indtryk , som Lalla Roohh , Thalaba , Roderick, Kehama og alle disse andre Digte efterlade . Det er store pyntelige Indretninger , der ere lempede efter Modens Fordringer . Geniets rette Kjendemærke er , at det formaaer at opdage en eller anden stor og hidtil ukjendt Egn i den menneskelige Natur , og dette Kjendemærke mangler hos dem ; de <b>vise</b> os blot , at de ere dygtige Folk med mange <b>Kundskaber</b> . I det Hele taget vil jeg hellere see Østerland

<h4>861</h4>paa Logik , og støttet til Frygt for Mennesker , til Tanken om Gud og Pligtfølelseu ; der er intet , der kan overvinde den , — det skulde da netop være en ensom Sammenkomst Klokken halv syv om Aftenen . Nu er Skridtet da endelig gjort , og den stakkels frygtsomme <b>Kvinde</b> bliver overrasket af sin forbittrede Mand , og i hvilken Situation ! Det maa man læse i selve Bogen . Nu vil hun da sikkert tie stille , skamme sig og græde , og den moralske Læser er ganske <b>vis</b> paa , at hun nu vil angre . Min kjære Læser , Du har ikke gjort Regning paa Instinktet og Nerverne . Imorgen vil hun være kydsk , men i dette Øieblik gjælder det om at blænde Manden , bedøve ham , forvirre ham , om at frelse Juan og at frelse sig selv , om at føre Krig . Naar Krigen er begyndt , maa man føre den med alle Slags Vaaben , og da først og fremmest med Frækhed og Sfrjældsord . Denne ene Tanke , dette Øieblikkets Krav , fortrænger alt Andet , det er derved en <b>Kvinde</b> er <b>Kvinde</b> , Julie her skriger , og det saa heit , som hun kan , det er et rent Regnskyl : Forbandelser og Beskyldninger , Spot og Trods , Besvimelser og Taarer . I Løbet af et Kvarter har hun faaet tyve Aars Erfaring ; en hæderlig <b>Kvinde</b> kan pludseligt , i et Øieblik , blive til en fuldendt Skuespillerinde, uden at man aner det , og uden at hun selv aner det . Man veed jo ikke , hvad man selv kan blive til . Man troer sig fornuftig og menneskelig , og det er maaske sandt for idag : man har spist til Middag og sidder i magelig Ro i et godt Værelse . Maskinen gaacr jevnt og glat , thi Hjulene ere smurte og de ere i Ligevægt , men lad os blive flyttede over til et Skibbrud eller et Slag , lad Blodmangel eller Overflod paa Blod blot et Øieblik forstyrre Maskinens Hovedhjul , og man vil see en Gal eller Idiot hyle og rave . <b>Dannelse</b> , Opdragelse , Eftertanke, Sundhed dække os med deres glatte , polerede Overflade, men lad os rive dem af en for en , eller alle paa een Gang , og vi ville lee , ved at see det Dyr , der ligger

<h4>936</h4>Legeme , hans Hjerte og hans <b>Aand</b> ere byggede og skikkede til noget bestemt , og af denne varige Ordning , som alle de tidligere Aarhundreder have bidraget til at styrke eller til at oprette , fremgaaer der stadige Ønsker eller Evner , der saa igjen bestemme hans Villie og Handling . Saaledes danner sig hos ham det ideale Porbillede , som fra nu af altid vil svæve ham for Øie , dunkelt eller klart , fuldendt eller i Udkast , som vil samle al hans Higen , alle hans Anstrængelser og alle hans Kræfter om sig , og som vil faae ham til at arbeide mod eet <b>Maal</b> Aarhundreder igjennem , indtil han tilsidst fornyes ved Afmagt eller Seier , og saa sætter sig et nyt <b>Maal</b> , og med nye Kræfter begynder paa en ny Bane . Den katholske og begeistrede Spanier betragter Livet med en Korsfarers , en Forelskets og en Ridders Øie , han opgiver Arbeide , Frihed ogVidenskab, og kaster sig efter sin Inkvisitions og sin Konges PJxempel ind i vilde Religionskrige , i romantisk Lediggang , i overtroisk og lidenskabelig Lydighed , i frivillig og uoprettelig Uvidenhed * ) . Lehnstidens theologiske Tydsker indeslutter sig villigt og trofast i sin lille Kreds , under sine smaa Fyrster , ifølge sin naturlige Taalmodighed og arvelige Troskab , han beskjæftiger sig med sin <b>Kone</b> og sin Husholdning , han er tilfreds med at have erhvervet religiøs Frihed , hans tunge Temperament holder ham tilbage i det plumpe legemlige Liv , og i den sleve Ærbedighed for alt det Bestaaende . Italieneren , der er rigere begavet og tidligere udviklet end alle de andre , men som mindre end nogen af de andre er istand til frivillig Tugt og moralsk

<h4>1906</h4>Herolder under den første Vaabenkonges Ledelse . Dommerne i deres Embedsdragter ventede for at give Raad om juridiske Spørgsmaal . Omtrent hundrede og halvfjerdsindstyve Lorder , trefjerdedele af Overhuset som Overhuset den Gang var sam- mensat , gik i høitidelig Orden fra deres sædvanlige Plads i Forsamlingen til Dommersædet . Den Yngste af dem gik foran , nemlig George Eliot , Lord af Heathfield , der nyligt var blevet adlet for sit mindeværdige Forsvar af Gibraltar imod Frankrigs og Spaniens Flaader og Hære . Den lange Række blev sluttet af Hertugen af Norfolk , af Rigets høie Embedsmænd af Kon- gens Brødre og Sønner ; Prindsen af Wales kom sidst , iøinefal- dende ved sin smukke Skikkelse og sin ædle Holdning . De gamle graa Mure vare betrukne med Skarlagen , hele de lange Galerier vare fyldte , og sjeldent har en saa udvalgt Tilhører- skare vakt en Talers Frygt eller Begeistring . Et stort , frit , oplyst og rigt Kongerige havde fra alle sine Egne sendt Ynde og <b>kvindelig</b> Skjønhed , <b>Kundskab</b> og <b>Aand</b> , Repræsen tanter for alle Videnskaber og alle Kunster til denne Hal . Der sad det brunsvigske Huses unge lyshaarede Prindsesser rundt om Dron- ningen ; der betragtede store Kongers og store Republikkers Gesandter med Beundring et Skue , som intet andet Land kunde <b>vise</b> dem Magen til . Der sad Siddons , i sin majestæ tiske Skjønheds fagreste Blomstring , og saa med Bevægelse paa en Scene , der overgik alt , hvad Skuepladsen kunde efter- ligne . Der tænkte det romerske Keiserriges Historiker paa de Dage , da Cicero førte Siciliens Sag mod Verres , og da Tacitus i et Senat , der endnu bevarede et Skin af Frihed . tordnede mod Afrikas Undertrykker . Der saa man Side om Side Tidens største Maler og dens største Lærde . Dette Skue havde lokket Reynolds fra det Staffeli , der har bevaret os saamange Forfat- teres og Statsmænds tankefulde Pander og saa mange ædle Da- mers fortryllende Smil . Det havde faaet Parr til at standse sit Arbeide i den mørke og dybe Grube , hvorfra han havde hentet saa stor en Skat af <b>Lærdom</b> , en Skat , der altfor ofte blev be- gravet i Jorden , og altfor ofte blev vist frem med überettiget og upassende Fordringsfuldhed , men som dog ligefuldt var kost- bar , solid og glimrende . Her saa man hendes vellystige Skjøn- hed , hvem Tronens Arving i Stilhed havde givet sin Tro . Der var ogsaa han , en skjøn Slægts skjønne <b>Moder</b> , den hellige Ce- cilie , hvis fine Træk , oplivede af Kjærlighed ogMusik , Kunsten har frelst fra den fælles Undergang . Der vare Medlemmerne af det glimrende Selskab , som bedømte og anførte Vers og vexlede vittige Svar under Mistress Montagues Paafugleforhæng , og her sad de <b>Damer</b> , hvis Læber besad større Veltalenhed end selve

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009020600097'target=_>, 1873, Læsebog for Folkeskolen og Folkehjemmet</a><h4>2251</h4>83. ( 44. ) Gange Rolv . Ragnvald MSrejarls Ssn Rolv var jaa fver og fsr , at ingen Hest kunde bere ham , og derfor kaldtes han Gange-Rolv . Han drog tidlig ud paa Vitingstog ; en Sommer hug han Strandhug ( rpvede Fe ) i Norge ; dette havde Harald haarfagre strengelig forbudt , og derfor blev Rolv utleg eller fredlss og maatte forlade Norge . Han rejste meb sit Fslge i mange Aar i Viking og herjede i England , Holland og Frankrige . Paa den Tid var Karl den enfoldige Konge i Frankrige ; han fandt det bedst at holde Fred med Rolv og bsd ham sit Venstab og et stort Stykke af sit Rige . Rolv gik ind herpaa , tog den kristne Tro og blev den franske Konges Jarl . Efter Stikken der i Landet stulde Rolv nu kysse Kongens Fod ; men det sagde han rent ud , at han aldrig vilde gjsre . For dog at vere rimelig , bsd han en af sine Hndmend at tysse Kongens Fod , og denne greb da fat i Foden og lsftede den op til sin Mund , faa at Kongen faldt baglengs , og stor Latter opftod . Rolvs larledSmme hed Nordmandi , og han og hans Mtlinger hed Rudejarlerne , fordi de boede i Rouen ( Ruang ) , forn kaldtes Ruda i vort <b>Maal</b> . Det norste <b>Maal</b> blev ilte lenge talt i Nordmandiet ; det var kun faa af Nordmennene , forn havde Hustruer med , og , da de siden tog Landets <b>Kvinder</b> tilegte , blev det franske Sprog BSrnenes Modersmaal . Men Nordmendenes Sans for Sagavefen og Skaldekunst havde Nordmcmnerne arvet , og de var de forste , forn ftrev Sagaer og kvedede i det franste <b>Maal</b> , og dette blev nu snart de Fornemmes Sprog rundt om i Europa og holdtes i stor Agt og Wre . Nordmannernes <b>Aand</b> og Ridderlighed blev et Msnster for hele Europa , og Riddervefenet havde sit egentlige Hjem i Nordmandiet . Siden grundede Nordmcmnerne et Rige i Neapel og Sicilia , og i Aaret 1066 tog de England , og Kongerne i England ere den Dag idag Gange-Rolvs ætlinger .

<h4>3958</h4>Mand ved at belegge sine Ord , nåar han stal indlede og fremfMe sit Mrinde . leg stal kun sige , at , dersom nogen lider af Heftighed og Fremfusenhed , saa skulde han fMe Lejlighed til at komme i Lag og omgaaes fortrolig med den norske Almuesmand for at lere Sindighed af ham . Pigen fra Mauranger har jeg kcmste talt med ; men da kom jeg itte Fortellingen ihu . Det var en <b>Kvinde</b> fra den samme Bygd , nu gift <b>Kone</b> og boende ien Nabobygd . Der var fmaa Kaar , og der stulde Anstrengelse og Omtanke og Forsigtighed til for at holde Mangelen ude . Men hun var <b>Sjelen</b> i Husholdningen , og jeg mener , at hendes æresfølelse og Arbejdsvane stal udrette store Ting . leg fer hende endnu , med det sint rynkede Ansigt ; hun er en af disfe mangfoldige Almueskoner , hvis Indsigter og Dyder jeg har lert at hFjagte , og forn jeg i Dagenes LB kan glede mig ved at mindes . Ja , der bor meget i den simple norske Mand og <b>Kvinde</b> , og det merkes paa deres Kundstab og <b>Dannelse</b> , at den sunds norfle Folte- Skik kan give Ungdommen en ypperlig Opdragelse .

<h4>4227</h4>nogle falste Lerere , forn fagde ham , at de kunde oplutte for ham de rette Dsrs for al Erkjendelse , saa at han vel stulde sinde nok for bestandig . I hele ti Aar drog han omkring med disse opbleste Mennester, disputerede om , hvad Gud er , hvad Synden er , kunde dog herved Me losrive sig felv fra Syndens Baand og erilerede tilsidft alt , hvad han hidtil vidste , for et stussende Blendverk . Under alt dette led hans <b>Moder</b> usigelige Hjertekvaler ; hun ssrgede over sin SM , som om han var dsd . Men , da hun klagede sin Smerte for Bistoppen i Menigheden , trsstede denne hende og sagde : „ En Ssn , for hvem saa mange BMner opsendes , saa mange Taarer udsses , kan umulig vere fortabt . " — Medens saaledes Augustinus lod sig henrive af ethvert Lerdoms-Vejr , erfarede han ret egentlig hos sig selv , hvad Apostelen Paulus siger : „ Kjsdet begjerer imod Aanden , og Aanden imod Kjsdet . " I Karthago var han allerede bleven Lerer i Veltalenheden og havde erhvervet sig et Navn blandt denne Verdens <b>vise</b> ; da sattede han den Beslutning at drage til Nom for der at stille fin Tsrst efter Kundstab ved at hsre nogle lerde Mend , hvis Bersmmelse var naaet frem til hans Pren . Hans <b>Moder</b> frygtede, at Hovedstadens fordervede Seder i Bund og Grund stulde sdelegge ham , og bad ham derfor indstendig at blive . Han lovede det ; men alligevel drog han afsted . Ved Efterretningen herom ilede hun ned til Strandbredden ; fjernt ude paa Havet kunde hun endnu sjne det Skib , der fsrte den fortabte SM bort fra hende ; da knelede hun ned og bad inderlig til den almegtige , at han dog vilde omvende hendes SMZ Hjerte eller da hellere lade ham omkomme paa Havet , end at han i Rom aldeles stulde gaa tilgrunde . Og Gud hsrte den fromme Moders BM . — Imidlertid for Augustinus over Havet og kom til Rom ; men hos hine Mend fandt han Me , hvad han fsgte . Da ilede han videre til Mailand , og her skulde Vendepunktet i hans Liv indtrede . Paa den Tid predikede Biskop Ambrosius i Mailand , og Augustinus hsrte ham flittig for hans Veltalenheds Skyld . Samtidigt hermed studerede han den gamle greske Filofof Platos Skrifter og sirebte at forene , hvad han heri fandt , med Ambrosius ' s Prediken . Den Herre Kristus begyndte Han alt at betragte forn en saare ovlyst Lerer ; men det hedensie , syndige Vesen opfyldte endnu fremdeles hans Hjerte . Da greb han endelig , beveget af Ambrosius , til den hellige Skrift , og under Lesningen Heras , fornemmelig Apostelen Paulus ' s Skrifter , blevs tilsidft hans Ajne opladte . Han erkjendte fig felv og sit forrige Livs Forfengelighet», da han vilde begribe Gud med sin Forstand , uden i 3)dmyghed at hengive ham sit Hjerte , da han vilde erkjende Gud , uden fsrst at tro paa ham og eiste ham . Under denne uåndelige Trengsel og Kamp hMe han fortelle om nogle fornemme Mend , forn havde opgivet fine Anesposter i den Hensigt ene at leve for Kristo . — Da sagde han med dyb " Bevegelse til en af sine Venner : „ Se ! disse ulerde Mend rive Guds Rige til sig , og vi , med al vor <b>Lerdom</b> , vi blive tilbage og leve i vore Synder ! " Han ilede ud i sin Have ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092303056'target=_>, 1865, Bibelen, eller Den hellige Skrift</a><h4>796</h4>Moder , det hele Folk , som bcrrer den almindelige Skyld , det hele Is-6 rael , " — Fordi der er ikke Forstand . Forstand n : Gudscrkjendelse . Sml . V . 1.  Der er ei Guds <b>Kundskab</b> i Landet . " Fordi du , mit Folk Israel . At du ikke j ? al gjsre pr. osv. Israels Folk var ved Pagtcn med Gud indviet til at vcerc et Folk af Prester ( 2 Mos . 19 , 6 ) . Hvad Prestcrne vare for Israel , det skulde Israel atter verre for alle andre Folk . Denne prestelige Vcerdighcd , som alene hvilcde paa Guds Vrkjendelse og Lovens lagttagelse , skulde det pagtesutro Folk miste og af Herren blive anseet og behandlet lige med Hcdningerne . — BDe aade .  De , " nemlig Presterne blandt Israels Folk . Disse vare sjunkne dybt , da de taaltc Kalucdyrkelscn i Bethcl og Dan- , og cftcrat de nn engang vare afvcgne fra Hovedbudet om ikke at gjore noget Billede af Herren , tog de det heller ikke saa noie med Ovcrtrcedrlsen af de andre Bud . Mit Folks Syndoffer . Ester Loven skulde Preslernc spise af Syndofferet ( 3 Mos . 6 , 19. 23. ) ; men dette var en hellig Handling , hvorved Presten sindbilledlig tog den Misgjerning , som var sonet ved Syudofferet , paa sig og ganske udstettedc deu i stn Persons Hellighed . ligesom i en hellig Ild ( sml . 3 Mos . 10 , 17. 16 ) . Israels uvocrdige Prester derimod fraadscdc i Syndoffcrc og ma ' skede sig med Lyst af deres Folks Misgjerninger . De oplsftede osv. lHebr .  o g rfter deres Misgjerning higer deres Sjcel " ) , fordi mange Misgjerninger bringe mange Syndufferc somend Syndoffernc i Virkeligheden ikke kunde ndstctte 3 ) lisgjerningerne , Hebr . 10 , 7. 11 ) . — 9 Derfor ligesom Folket , saa osv. Som Folket , saa Presterne ! Som Prestcrne , saa Folket ! Folket vil ikke af Prcsterue lcere at kjende Gud og hans Lov ; Prestcrne selv bryde sig ikke om at bcstddc og meddele Gnds Erkjcndelse . Som Begge synde i Fellesstab , stulle de og straffes io i Fellesstab . — Og de ji ' ulle crde og ikke osv. Daarcrne mene endnn den Dag idag : Meget hjelper meget ! Megen Mad erncrrer rigelig , megen kjodelig Omgang mellem Mand og <b>Kvinde</b> bringer mange Born , formerer , ndbredrr og gjor Folket moegtigt . Men Skriften lcerer her og overalt det Modsatte . Meget med Synd bringer blot Skade ; Lidet med Guds Velsignelse boerer megrn Frngt . Ps , 128. Erfaringen stadfester dette , og saavel den som Guds Ord gjendrivcr daglig de i i Vantroes materialistiske Vildfarclser . — Most . Most , som var begyndt at gjcerc , var en meget yndet Drik og berusede ligesom Vin ; af denne sin Virkning har den sit Navn paa Hebr , Sml . locl 1 , 5. — 12 Sit Trer , Nfgudobillede af Troe . Sml . E 40. 20. ler . 2 , 27. 10 , 18. Maaske stal herved forstaacs Terafim ( C . 3 , 4. ) , GudcbiNcdcr af Troe , som Sanosigerne betjente sig af ) . Og deres Rjep j ? al forkynde den » det ( <b>vise</b> dem , hvad de skulle gjore ) . Kjcppe saavelsom Pile sEz . 21 , 16. ) brugtes hyppigt af Tcgntydcrne , isoer hos Chaldceerne. Man tog 2 Kjeppe i Haanden og stap dem paa Maa yg Faa ; cftersom nu Kjeppene faldt tilvenstre eller tilhsire , foran eller bag , saa man deri et gnnstigt eller ngunstigt Tegn . Der omtales ogsaa andre Maader , hvorpaa man spaaede ved Hjelp af Stave . Horeri , lg Wgteskabsbrud , Pagtesbrud . — Derfor bedrive eders Dsttre Hor , og osv. Enten fordi Afgudstjenestens aandeligc Utugt har fordcervct den jomfruelige Blnfcerdighed og Hustugten overhovcdet ; eller fordi Herren straffer Israel for dets pagtesbrud med at lade Fienden som

<h4>1937</h4>ssml . 1 Kong . 7 , 6. ) . saalcdes kunde del den Hentydning , vi her finde til Tcmpelhallcn , indcholde et Vink , om . at dette Edikt ndgaar fra Gud , som den Konge , der selv dommer sit Folk . — Forbandelse o : Gnds Fordommelsesdom , som fornd udtales over enhuersomhclst Ovcrtrcedcr af Loven , Son , udgaar over det ganske L. and . Meningen er ikke den , at Dommen gjelder og stal ramme det bele Land , men at den stal blive bckjendt i hele Landet , fornt Ingen stal knnne undstylde sig med sin Uvidenhed . Hver den , son » stjcrler hver dm , som svcrrger falskelig . Tyve og Mencdere anforcs her ei , ' cmpelvis , de Forste som Enmpcl paa Ovcrtrcrdcrc af Lovens anden Tavle , de Sidste som Ezcmpcl paa Overtredere af Lovens forstc Tavle . Disse stnlde fnldkommen ndryddcs af det hellige Folk , saa der ikke bliver Spor tilbage af deres Boliger sV . 4 , ) . , Dcnncd er ogsaa Ud > ryddelscn af deres hele Slcrgt forbunden ; thi den Ugudelige faar ikke beholde nogen Arvclod i Landet . — Og Engelen — gik ud . For liavde han vcrret stjult , usynlig , nn aabrnbaredc han sig synligcn ; dette siges her , fordi han nu griber virksoint ind i den Handling , som Synet fremstiller . Sml . 2 , 3. Den samme Betydning saabenbare , <b>vise</b> sig ) har Ordet  gaa nd " iV . ti og 9. — En Gfa . Efacn . inrst Homcren Hebrcrerncs storstc <b>Maal</b> for torre Varer somtrcnt 2 ^ norst Skjeppe ) , er her valgt paa Grnnd af sin Fonn og Storrclse . Her at tcenke paa et falskt <b>Maal</b> eller ogsaa paa et fnldt Syndemaal . gaar neppe an , da Terten ikke indcholdcr nogen Hentydning hverken til det Ene eller det Andet . Deres Dienicerke . Hcbr .  deres Die . " Dette kan forklares deres Skikkelse , deres Udsccnde ; altsaa : dette er deres Udseende i alt Landet sdct hele Land ) , saaledes se de nd ; denne Efa er et Villede paa dem . Men ViUede paa hvem ? Dette kan ikke vcere Andre end Tyvene og Mcncdcrne , forn ere nwvnte i det Fore » gaacndc IV . 3. og 4 ) . Dog faar dette forst sin fnldc Betydning , naar man ikke tamker paa Efaen alene , men tillige — hvorom der tales i det Folgknde — paa hvad Efaen indcholder . paa <b>Kvinden</b> , som er Villede paa Ugndelighcdcn sV . 7. 8. ) . Meningen bliver da : Efacn . som indrstntter <b>Kvindcn</b> . Ugudelighedcn , er et Billede paa alle de Synder. som stulle ndryddes . — Andre oversatte ssom vor Bibelovers . ) :  Dette er deres Hensigt sderes Streben ) i hele Landet . " Meningen vilde da vcerc : . Syndernes Strceben ganer ild paa at gjore sine Synders <b>Maal</b> fuldt og saaledes nedkalde over sig i fuldt <b>Maal</b> Guds Straf . Men denne Fortolkning har , uanscet mcgct Andet , iscrr det mod sig , at Efaen her slet ikke er Etraffcmiddel , men blot et Kar . hvori Ugudelighcdcn stal fores bort fra , det hellige Land til det Land , hvor dens Hjem er . — Stedet er dunkelt . — Det folgcnde V . ovcrscrttes forssjclligt . Enten sum vor Vibelovers . : der  oploftcdes shcrvcde sig ) et Ccntner Vly ; " Meningen er da , at den Blystive , som lan over Efacns Aabning , hcrvede sig , fornt Profeten kunde fa a se , hvad der var i den . Eller ssaalrdes Schmiedcr , se Indlcdningen til C . 5. ) :  der svcevcde et Ccntner Bly . " Meningen er da ikke , at Cfaens Aab > ning for var tildcrkkct af Blyet , men at Blyet fra forst af svcevede , ved Siden af Efaen , forat Profeten kunde have det ved Hamidcn naar ? ! han stnldc tildcekkc Aabningcn paa Efacn og saaledes hindre Ugndcligheden fra aNstippe nd , forcnd den var kommen til sit Bestem »

<h4>2375</h4>Boger i det ' G . T . Men allerede de Uoverensstemmelser med den virkelige Historie , som ftndes i 1. Cap . , bevise dens Ucegthed , , hvad ogsaa Hieronimus har anerkjendt . Baruch , Son as Neria og Profeten leremias ' s Skriver , der sammen med denne drog til Wgypten ester Jerusalems Ddelceggelse ( ler . 43 , 6. ? . ) opholdt sig , fortcelles der os strax i Begyndelsen as denne Bog , i det ste Aar ester Templets Ddelceggelse i Babel og lceste der denne Bog for Kong lojachin og de ovrige fangne Joder paa den Faste- og Bededag , som holdtes til Minde om Templets Ddelceggelse . ( Denne Fastedag holdtes paa den 7 de Dag i den ste Maaned , paa hvilken Nebukadnezars Hcerforer Nebusar-Adan var kommen til Jerusalem , forat opbrcende Templet . Sml . ogsaa Sach . C . ? . ) . De Tilstedeværende samlede Penge og sendte dem tilligemed de hellige Kar , som Fienden havde bortfort til Babel , til Presten lojakim i Jerusalem . I disse Angivelser vrimler det af Anstod mod den virkelige Historie ; thi i det sdelagte Jerusalem var dengang ingen Prester og intet Tempel , og der offredes efter Templets Ddelceggelse ikke i Jerusalem , men i Mizpa ( ler . 41 , 5. 6. ) ; de hellige Kar bleve forst meget sednere sendte til Jerusalem as Persekongen Cyrus ( Csr . 1 , 7 — 11. ) ^ ) . Sammen med Pengene , som skulde anvendes til Offere , sendte de Fangne ^ denne Bog af Baruch og befalede , at den skulde lceses i Templet paa Festdagene og Aarsfesterne ( Sabbathen og Nymaanedagene ) . Denne historiste Indledning ( 1 , 1 — 14. ) er Bogens svageste Del ; af dens videre Indhold se vi , at den er skreven af en Mand , der var fortrolig med Guds Ord og det G . T . ' s <b>Aand</b> , uden at vi dog ere istand til at bestemme den Tid , Bogen er affattet . Bogen indeholder forskjellige Afsnit , som ere knyttede til hverandre uden Forbindelse : 1 ) En Bodsbsn af det nedboiede Folk under Fangenfkabet ( C . 1 , 15 — 2 , 35. ) . 2 ) Bon om Befrielse fra den velfortjente Straf ( 3 , 1 - 8. ) . 3 ) Cn Formaningstale til Israel om at ssge den sande <b>Visdom</b> og omvende sig til den ( 3 , 9 - 4. 4. ) ^ ) . 4 ) En Trostetale af Zion til hendes fangne Born ( 4 , 5 — 29. ) . Med Zion menes her Jerusalem i <b>aandelig</b> Forstand , de Troendes <b>Moder</b> , Jerusalem heroventil ( Gal . 4 , 26. ) . 5 ) Trsstetale til selve Jerusalem ( 4 , 30 — 5 , 9. ) . 6 ) Det saakaldte leremias ' s Brev ( C . 6. ) .

<h4>2824</h4>Denne Bog taldes Salomos Visdoms Bog , som om denne kongelige Vismand selv havde strevet den , forat undervise Fyrster og Regenter. Dog maa dette blot ansees for en digteriff Indklcedning , uden at dermed er tilsigtet nsget Bedrag , da der i Bogen , som er forfattet paa Grcesk , itte er nogensomhelst Antydning til , at det har vceret Forfatteren magtpaaliggenoe at give det Skin af , at den var forfattet af Salomo . Saavel Sprog fom Tanter lede os til at antage , at den er forfattet af en Isde , som levede i Wgypten under Ptolemceus Fyston ( mellem 145 og 116 f . Chr . F . ) , som var fortrolig med den grceske Verdensvisdom og til Guds og hans Folks Pris og i Modscetning til Hedenskabets Forvildelser viloe fremstille den sande <b>Visdom</b> , som udspringer af Gudsfrygt og Lydighed mod Guds Lov . Medens Jesus Sirach, som leuede i Palcestina , oprindelig blot sirev for sine Landsmcend , er denne Bog anlagt paa ikte alene at lcrses af Isder , men ogsaa af Hedninger . Forfatterens Navn er übetjendt ; men vi ville dog ikte undlade at omtale en Gisning , som er fremsat af en nyere Lcerd , om at Bogen stulde vcere forfattet omkring i36 af den alexandrinske Isde Aristobulus , som var Lcerer for de to kongelige Brsdre : Ptolemceus Filometer og Pt . Fysion . Vi finde her det samme Visdomsbegreb , som i det G . T . ' s hellige Skrifter og Jesu Sirachs <b>Visdom</b> . Sml . Hiob C . 28. Ordspr . C . 8. og 9. Snach 1 , 8. 6 , 19. 14 , 20. 21. 24 , 1. <b>Visdommen</b> tcrnkes som et selvstcendigt Vcesen , som i sig eier Ertjendclsen af Grunden til og <b>Maalet</b> for den hele Skabning , som har baade Vilje og Magt til ved hensigtsmccssige Midler at indrette Alt i Overensstemmelse med Guds Plan og fsre det til sit <b>Maal</b> . Ja , hele dens Vcesen bestaar itke i noget Andet end i denne <b>vise</b> Erkjendelse og Handlen . I denne Betydning betragtes den nu baade i Forhold til Menneskene og til Gud . I Forhold til Menneskene er den ligesom en <b>Moder</b> og Opdragerinde , ja ligesom en Brud , som hengiver sig i hellig Kjccrlighed til dem , som begjcere den , og meddeler dem sit Vcesen , saa de ved Retfcerdighed og Kydsthed naa til Crkjendelse af alle Ting og faa det evige Lw . I Forhold til Gud er den det fsrstskabte Vcesen , som fuldfsrer Skabelsen og staar ham bi i al hans Gjerning . Den fremtrceder altsaa ligeoverfor Gud og Menneskene som en Person , som har et selvstcendigt Liv , og der tales oftere om den som en saadan selvstcendig Person . Men da paa den anden Side denne Personlighed Me har noget andet Indhold end at vcere <b>vis</b> og gjsre <b>vis</b> , saa kan man slet ikte fastholde dens Personlighed , men denne svinder hen for os , saasnart vi ville fastholde

<h4>2897</h4>Verdensorden . Men bnade Tegn og Undere ssnUe tjene ru at aabcnbare den <b>Vise</b> , hvad Tider og Stunder der forestaar , Tider og Stunder .  Tid " angiver knn det bcstcinte Tidspunkt ,  Stund " sTimc ) dette Tidspunkts Beskaffenhcd- Undertiden knn Tidens Beståsfcnhed vcere angivet , ndcn at Tidspunktet angivcs noiagtigt . Sanledes i de fleste Spaadounne hos Prufcterne . — Men jeg var et Barn af god Art og havde faaet en god Sjcel .  En god Sjcel , " fra jeg ble » undfangcn i Moders Liv . sSml . Jesu Sir . 1 , 12. Anm . ) . Men Jesus siger :  3 ngen er god nden Vn , nemlig Gnd " sMatth . 1 l ^ ) , 17 , ) .  Uden Nogen fodcs paany af Vand og <b>Aand</b> , kan han ikke se Gnds Nige " sloh . 3 , Z . ) . Hvorledes kan da den opdigtedc Salomo her sige , at han er fodt med en god Sjcel ? Han bekender jo selv , at han er et dodeligt Menneskes ? , 1. ) , og ved , at Doden er kommen ind i Verden ved Djcevelens Avind ( 2 , 24. ) . Hvorledes kan han da fra Aodselen af havt en god d . e . syndefri Sjcel ! David , den historiske Salomos Fader , beviducr , at han er fodt af syndig Sced og at hans <b>Moder</b> har nndfanget ham i Synd sPs . 51,7 . ) , og ser nctop i denne Crkjendclsc den skjulte Sandhed , den hemmelige <b>Visdom</b> ( Ps . 51 , 8. ) . Hvorledes sknlde Davids Son have havt en ren god Sjcrl fra Fodsclen af ? De-n opdigtcde Salomo erklcrrcr , at Forcrldrcncs Kydskhed er af hoiestc Betydning for Borncne ; men David og Vathseba , Salomos <b>Moder</b> , havde allerede for dennes Fodscl avlet en Son i Hor . Hvorledes kan da den opdigtede Salomo sige , at den historiske Salomo har havt en god Sjcel fra Fodselen af ? Alt dette er ganske sandt . Men vi maa " lcrggc Mcrrke til : 1 ) at den gode Sjcrl . som her tillcrgges Salomo som " Barn , ikke er en Sjcrl , som er fri for den almindcligc Syndighed , men en Sjcrl , hvori der af Guds Naade var medfodte kraftige Anlcrg til hellig Strcebcn , saa han allerede tidligcn folte sig hendragen til Gud og Viodommrn . ' Dette stnar ikke til at nrgte ; tln 2 Sam . 12 , 24 - 25. hedder oet :  Og hnn lßathseba ) fodtc en Son , og han sDavid ) kaldte hans Navn Salomo ; men Herren elskede ham . Og han sendte Bnd ved Nctthan , Profeten , og kaldte hans Navn Icdidjah ( den Elskede , Guds Ven ) , fordi Herren < clskrde ham ) . " 2 ) Denne Salomo er et , omend nfuldkomment , Forbillede paa den fnldkomne Davids Son , og som saabant fremstilles og betragtes han ogsaa af Profetcrne . Saalcdro er han nn ogsaa her et Forbillcde paa den sande Davids Son , og derfor skildres han her nden dr Pletter , fom virkelig fandtes hos den historiske Salomo . — Det Vidncsbyrd om en god Sjcrl , som her gives Salomo , grnnder sig , som sagt , forsaavidt Forbillcdet niaa have et historist Tilknytningspunkt , paa 2 Sam . 12 , 24 — 25. Mange uyerc Thcologer mene vistnok , at Forfatteren her er inde paa den platoniske Lcrre om Sjcelcnes Prcecz-istens , og at det er hans Mening , at Sjcelen har renset og gjort sig god under et tidligere Liv her paa Jorden . Men da knnde det ikke hedde . at han havde faaet en god Sjcel . men at han havdc havt , medbragt eller erhvervet sig en god Sjcrl " ) . Ifolge Skriften beror dcrimod Sa-

<h4>3070</h4>frelse sig ued et fortvivlet Udfald . Herodes vendte sig ftrst mod Galilcra , hvor han tidligere haude va ' ret Statholder og det fariseiske Parti aldrig haude havt nogen synderlig Indflydelse . Han samlede snart en liden Hcrr ; men f ^ rend han kom sin Broder tilhjelp , ouerrumplcde han f ^ rst Byen lopve , hvorved en as de vigtigste Pladse faldt i hans Hcrnder ; derpaa undsatte han Masada . Nu fulgte en lang , induiklet Krig , i hvilken Josef paa Grund af sin Uforsigtighcd blev slagen og dra ' bt , og Hcrodes ved Hjelp af den romerske Preses i Syrien , Sosius , erobrede Jerusalem , efterat han haude beleiret den i sex Maaneder. Antigonus blev tågen tilfange , og Antonius lod hain paa Herodes ' s indtrcrngende Bpn , som ikke troede sig sikler , saalcrnge en af det makkadanske Hus levede , henrette ( 37 f . Chr . F . ) . Nu forfulgte Hcrodcs alle sine tidligere Fiender i Jerusalem ; han tog Hevn over alle de Medlemmer af Synedriet, som havde villet straffe ham i hans Ungdom ; kun den frimodige Sameas skaanede og arcde han for hans Kjwkhcd og Ucgennyttighed . Nu havde han ikke mere at frygte nogen Medbeiler til Thronen . Han indbfld den sidste Makkablrerfyrste, den gamle Hyrkan , som endnu levede i Fangenskab blandt Partherne, til sig og gjorde ham alslags Lafter ; Hyrkan kom , og omcndstjont han hverken var Konge eller kunde verre Ippersteprest , da han var lemlestet , viste Herodcs ham dog al mulig Heder . For ikke < it have nogen Ippersteprest af det fariscriske Parti , som kunde volde ham Vanskeligheder i leusalem , hcntede han en Ipde af yppcisteprestelig 3 Et , ved Navn Ailllnel , en ringe übckjendt Mand , fra Babylon og lod ham salve til Ippersteprest . Og saa syntes da Herodes paa alle Kanter at have grundlagt og befestet sin Magt . Men denne hvilede alligevel paa en hul og strMelig Grund . Fremmed i sit Indre for alt det , som var dct Dyrebareste for hans Folk , støttende hele sin Magt til det , som udenfra skulde paatvinges loderne , folte Herodes sig aldrig tilfreds i fin Kraft og Virksomhed . For det Gode , som han tilsigtede og udfjslte , takkede Ingen ham , og det Onde , som han gjorde , fremstilledes endnu slemmere , end det virkelig var . Hvad der desuden lammede hans Kraft og ceggede ham til Vrede , var hans Familjcforhulde , som bragte ham i allehaande übehagelige Sammenstpd med den makkabceiske Fnmilje ; det var , som om dette synkende Hus hevnede sig paa Usurpatoren ved , alt i sine sidste Dpdstr<rkninger, at drage hani med sig i Faldet . Han fMe , at han haude ranet Kongestolen fra Makkabcrerne ; men han vilde gjerne give det Udseende af , at han som Hyrkans Slcegtning , som gift med Mariamne , haude en <b>vis</b> Ret til Kongevcrrdigheden , og gjorde sig derfor Umage med at hcrdre Hyrkan og holde ham i HEre ; men han kunde dog ikke skjule for sig selv , at hans Magt ikke hvilede paa hans Arveret , men paa > Romernes Gunst . Her var en svag Side , mod hvilken hans Fiender kunde rette og virkelig ogsaa rettede sine Angreb ; fra denne Side s ^ gte saavel det forbittrcde Folk som de sidste Makkabcrcre at komme ham tillivs , og der udspandt sig en Vcrv af afskyelige Roenker , som Herodes af al Magt s ^ gte at tilintetgjøre , hvad der dog ikke lukkedes ham , hvor skjcrndigt og grusomt han end bar sig ad . Hans skjenne Gemalinde Mariamne , som han elskede lidcnskabeligt , haude arvet hele sin Slcrgts stolte Sind ; hun kunde derfor aldrig glemme , at Thronen egentlig tilkom hcndes Slccgt , og foragtede i fit Hjerte Herodes . Hun haudc en Broder Aristobulus , som v « zr den ncermeste Arving til Ippcrsteprestembedet . Alexandra , Aristobulus^ og Mariamnes <b>Moder</b> , en overordentlig stolt og argjerrig <b>Kvinde</b> , som ikke bceuede tilbage for de skjcrndigste Midler , naar de kun forte til <b>Maalet</b> , vidste at vinde Antonius og Kleopatra for denne skjsnne Ingling . Man trcrngte fra alle Sider ind vaa Herodes med Spl < rgsmaal om , hvorfor han havde berpvet en saa herlig ung Mand den Vcrrdighed , som tilkom ham . For at hjelpe sig ud af Kniben , sagde han , at han kun forelMig haude indsat Ananel , indtil Aristobulus blev voxen . Endelig afsatte han virkelig Ananel , og den kun IVaarige Nristobulus blev Ippersteprest . Da nu Aristobulus for fyrste Gang forrettede sit Embcde paa LjZvsalernes Fest , blev denne sidste ling af det makkabceiste Hus overalt modtagen med lydelig Jubel . Da besluttede Herodes strax at rydde ham afveien . Han indbod ham til en Fest i lericho, som seiredes med Lege og alfkcns Legemsøvelser , og da en Del af Selskabet efter Dagens Hede badcde fig i en Dam , blev Aristobulus as nogle Hoffolk efter Herodes ' s Befaling holdt saalcrnge under Våndet , til han var

<h4>3104</h4>Thi Albinus hande dog idetmindste saainegen Skainfolelse , llt han Fuede sine Forbrydelser i det Skjulte ; men Florus brf « d sig ikke engang om at skjule sine Skjendselsgjcrninger , han gjorde sig til deraf , og behandlede Folket som om Keiseren ikke havde sendt ham forat styre det , men fornt ove Skarpretterembcdet paa dem som dMte Misdedere . At plyndre Enkelte tyktes ham et tjedsommeligt Arbeide ; han plundrede helst Byer og hele Distrikter , og lod Rpuerne have fuld Frihed , naar de blot gav ham Andel i Nyttet . Hele Egne foruandledcs til ørkener , ug en stor Mengde I ^ der uduandrede til Nabolandene . Ccstius Gllllus var dengang Statholder i Syrien . Da man var sig fuldt beuidst , hvor vllnskcligt det var at faa Ret mod Florus hos de romerste Embedsmend , da der stode saamange Midler til hans Naadighed , hvorved han kunde fordreie Sandheden , vovede Ingen at anklage Florus for hans Overordnede. Men da Gallus i Aaret 65 kom til Jerusalem til Paaskefesten , gik en stor Skare paa flere Tusinde ham im ^ de og bad ham forbarme sig over Folket og beskytte dem mod den rasende Florus . Denne stod under Forhandlingen ved Siden af Gallus og fogte nt gj ^ re Indernes Klager latterlige , men Gnllus beroligede Folket og louede at sFrge for , at Florus for Fremtiden regjerede med mere Maadehold . Derpaa vendte han tilbage til Antiochia , og Florus , som lcdsagcde ham til Kefarea , vidste underveis ved sine Lpgne og Bakvaskelser at indtage ham mod loderne . Fra nu af synes Florus at have haut den bestemte Plan for H2ie , at ophidse Inderne til Krig mod Romerne ; thi paa anden Mande saa han iugen Udvei til at kunne skjule sine Valdsgjerninger, men paa denne Maade kunde han maaske fan en erefuld Plads blandt de store tzerff < rere og faa Del i Byttet . Det var dengang e « meget tung Tid for Romerriget . Det var , som om Ondskabens og Fordervelsens <b>Maal</b> ikke alene var f ildt og maatte gaa over i Palestina , men overalt ; det syntes som om alle Vaand vare oplpste . Man kunde tro , at Dommens Dag allerede nermede sig for hele Verden . Det er HM mertuerdigt , at paa samme Tid , som den jødiske Neligions- og Statsforfatning gik tilgrunde , en Rystelse gik gjennem den hele Verden , som ligned ? den Solformørkelse og det Jordskjelv , som fandt Sted ved Jesu Dpd . Naar man betragter denne Tid , kan man vere i Uuishcd om man stal undre sig mest over den Frekhed , hvormed Lasten fremtraadte , eller over Menneskenes Taalmod . Oucr det romerske Rige herstede dengang Keiser Ncru . Han havde faaet en god Opdragclse , var udrustet med gode Anleg , men var for tidlig , allerede i en Alder af 17 Aar , bleuen Keiser ; hans skjendige <b>Moder</b> Agrippinas Hcrstesyge og en Har af Smigrere fordervede ha , n snart ; Udsvevclser og afskyelige Kvinderenker ved hans Hof gjorde ham endelig til et grusomt , stamlost Uhyre , som stred frem fra den ene Udaad til den anden , indtil han endelig lod sin egen <b>Moder</b> drebe . Dog nu , da den forferdelige Gjerning er fuldbragt , og Samvittighcden vaagnct , da han er i Nuished om , hvorledes han skal <b>vise</b> sig for sine Nndersaatter efter en faadan Nedselsdaad , da kommer det slaviske Folk og det endnu elendigcre Senat hain festlig smykket im ^ de , de modtage ham som i Triumf ! Forat dpue fin Samvittighed , styrter han sig i de vildeste Udsva > velser ^ hans gamle Lccrer Seneka forlader Hoffet , og hans Undlinge tanke blot paa , at tilfredsstille hans Lidenskaber og Luner , Han bliver mere og mere en Slave af alskens Vellyster , den utrolige Mselhed gjor ham mere og mere gridsk , hans Luner forlede hain til de daarligste Handlinger; han udraaber : FM mig har ingen Fyrste vidst alt det , han formnar — indtil han setter Ild paa Rom , gjcnnembrydcr Vjerge og indenfor Keiserboligen i en vandlos Egn anlegger en S ^ , som kunde ruinine ' en Flaade , og Bjerge , Skove og Gresgangc , forfnmlcr Senatet og forhandler for Aluor med det om en JOgtcskabskontrakt inellem sig og tvende Mend , ja indtil han endelig optreder offentlig som Sanger og Skuespiller , og drager omkring og beiler om den kunstelskende Mengdcs Bifald i Nom og Grekcnland ! Hans Historicstriuer Tacitu > > sinder det tilslut sna modbydeligt at berette om alle hans Grusomhcdcr , at han siger ! ( 16 , 16. ) : Om jeg end fortalte om Krige blandt fremmede Folkeslag og om hvorledes romerske Borgere have osfret Livet for Fedrelandet , vilde jeg blive tret deraf og frygte for at trette Leseren , som ikke haude Lyst til at lMe om huorlcdes saamangc Borgere , den ene efter den anden , have faaet en sørgelig , omcnd erefuld Dod . Men hvor aands-

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008110603028'target=_>, 1848, Vor Herres og Frelsers Jesu Christi nye Testamente</a><h4>1085</h4>deres Stad var bygget , for at styrte ham ned . 30. Men han kom igjennem midt imellem t « m og gik bort . Joh . 8 , 59. 111. 31. Og han kom ned til Capernaum , en Stad i Gali , lcra , og lcerke dem paa Savbaterne. Match . 4 , 13. 33. Og de forundrede sig saare over hans <b>Lcrrdom</b> ; thi hans Tale var medMyndighed . Matth . 7 , 28. 29. Mare . 1 , 22. 33. Og der var et Men , neffe i Synagogen , som hav , de en ureen Djcrvels <b>Aand</b> , og raabte med hoi Nsst Marc.l , 23 fg . 34. og sagde : Eja ! hvad have vi med dig at staffe , Jesu afNazareth ? Er du kom , men for at forderve os ? Jeg kjender dig , hvo du er , den Guds Hellige . 35. Og Jesus truede ham og sagde : Tie og far ud af ham ! Og Djcevelen kastede ham midt iblandt dem , og foer ud af ham og gjorde ham ingen Skade . 36. Og der kom en Rcedsel over Alle , og de talede med hverandre og sagde : Hvad er dog dette , at han byder de urene Aander med Myndighet » og Magt , og de fare ud ? 37. Og Nygtet om ham udspredtes allevegne i det omkringliggende Land . 38. Men han stod op og gik fra Synagogen ind i Simons Huus , og Simons Husirues <b>Moder</b> var plaget med en svar Feber , og de

<h4>1984</h4>kobus , Alphcei Son , og S mon ' Zelotes , og Judas lakobi Broder . Matth . 40 , 2 - 4 * d . e . den Nidkjcrre . 14. Disse holdt alle een drcrgtigen ved i Bsn og Paa kaldelse tilligemed Qvindern og Maria , Jesu <b>Moder</b> , og med hans Brsdre . c . 2 , 1 111. 15. Og i de Dage stod Petrus op midt iblandt Disci plene og sagde : — men de var en Skare af henved hun drede og tyve Personer tilsam men , — 16. I Mcend , Brodre ! det burde de Skriftens Ord at fuldkommes , hvilke den Hellig- <b>Aand</b> havde forud talet ved Davids Mund , om Judas , som blev deres Veileder , der greve lesum . Ps . 41 , 10. Joh . 43,48 ; 48 , 3. 17. Thi han var regnet med os og havde annammet dette Embedes Lod . Lue.6 , 16. 18. Denne altsaa forhvervede en Ager ved Uretfcerdigheds Lsn , og der han styrtede ned , brast han itu , og alle hans Indvolde udvcrltede . 2 Sam . 47 , 23. Matth . 27 , 5. 19. Og det er bleven vitterlige for alle dem , som boe i Jerusalem , saa at den Ager kaldes paa deres eget <b>Maal</b> Hakeldama , det er Blodaaer . Sach.4 4,43 . Matth . 27 , 7. 8. 20. Thi der er strevet i Psalmernes Bog : Hans Bolig blive sde , og Ingen vcere , som boer derudi , og en Anden annamme hans Embede . ' Ps . 69,26 . Ps . 409.8 . 21. Derfor bor det , at en

<h4>2301</h4>as dem , og af hcederlige grceste <b>Qvinder</b> og Mcend ikke Faa . 13. Men der Isderne af Thessalonica fik at vide , at Guds Ord blev og i Beroea forkyndt af Paulus , kom de og derhen og satte Folket i Bevcegelse . ITHess.2 , l4 . 111. 14. Men sirax sendte da Brsdrene Paulus bort , for at drage henimod Havet ; men baade Silas og Timotheus bleve der tilbage . c.16 , 1. 15. Men de , som ledsagede Paulus , fsrte ham indtil Athenen;' og der de havde faaet Befaling til Silas og Timotheus, at de med det snareste stulde komme til ham , droge de bort . ITHess.3 , l . 16. Men der Paulus forventede dem i Athenen , harmedes hans <b>Aand</b> heftig i ham , der han saae Staden at vcere saa afgudist . 17. Han talede derfor l Synagogen til Isderne og de Gudftygtige , og paa Torvet hver Dag til dem , som han traf paa . 18. Men nogle Philosopher , Epicurceer og Stoiker , disputerede med ham ; og Nogle sagde: Hvad vil denne Ordgyder sige ? men Andre : Det synes , han vil forkynde fremmede Gu , der ; fordi han forkyndte dem ved Evangelium lesum og Opstandelsen . 1 C0r.4 , 12 19. Og de toge ham og fsrte ham op til Domsiedet og sagde : Kunne vi faae a vide , hvad dette er for en m <b>Lcerdom</b> , som du taler ?

<h4>3508</h4>dere , Ryggeslsse og Vanheli lige , Fadermordere og <b>Moder</b> , ' mordere , Manddrabere , Gal . 5,23 . Norn . 1.26 fg . Gal . 3 , 19 ; 5,19 fg . 11 ) . Skjsrlevnere , Syndere imod Naturen , Mennestetyve , Lsgnere , Meenedere og hvad Andet , der er imod den sunde <b>Lcerdom</b> , 2 M05.21 , 16. 11. efter den salige Guds herlige Evangelium , som mig er betroet . c.6 , 15.1 The55.2 , 4. 12. Og jeg takker Christum lesum , vor Herre , som haver gjort mig stcerk , at han agtede mig at vcere tro og satte mig til Embedet , Ap.G .9 , 15. 13. mig , som tilforn var en Bespotter og en Forfolger og en Forhaaner . Men mig er Barmhjertighed vederfaren , thi jeg gjorde det uvitterligtiVanl tro . Ap.G .8 , 3 ; 9,1 . Phil . 3. 6. Av . G . 3. 17. 14. Men vor Herres Naade har viist sig overvettes stor med Tro og Kjcrrlighed i Chrii sto Jesu . Norn . 5,20 . 15. Det er en trovcerdig Tale og aldeles vcerd at an ^ nammes , at Christus Jesus kom til Verden for at gjsre Syndere salige , iblandt hvilke jeg er den sisrsie . c.4 , 9. Mat1h.9 , 13 ; 18,11 ; 1,21 . 16. Men derfor er mig Barmhjertighed vederfaren , for at Jesus Chrisius vilde paa mig , som den storsie , <b>vise</b> al Langmodighed , dem til et Exempel , som stulle troe paa ham til det evige Liv . v. 13. Mare . 16,16 . 17. Men den evige Konge ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014060208013'target=_>Wexels, W.A., 1839, Forklaring over Det nye Testamentes Skrifter</a><h4>347</h4>B . 16 - 19. Jesus opgiver ikke Tanken om , med den himmelske Sandhed at komme hendes <b>Aand</b> noermere , end han hidtil synes at vcere kommen ; kun soger han ad en anden Vei at naae det samme <b>Maal</b> . Idet han synes aldeles og med Eet at afbryde , afbryder han dog ikke i Grunden , vedbliver meget mere med uafbrudt Kjoerlighed og Taalmodighed at forfolge sin Hensigt : at voekke Qvindens <b>Aand</b> baade til <b>Bevidsthed</b> om og til Lcengsel efter " den levende Vandkilde " , hvorom han havde talet . Han kaster et Blik ind i hendes Liv , fremstiller dette for hende i de Retninger , i hvilke hun troede det skjult for menneskeligt Sie , og fremkalder derved hos hende Tanken om , at han maa besidde meer end menneskeligt Syn , at han maa voere begavet med prophetist Seer-Blik . Jo mere hun nu lcerte at forstaae ham , og jo mere hendes Tanke lededes ind i det <b>Aandeliges</b> Gebeet , des mere opgik naturligviis ogsaa Lyset for hende over de Ord , hun havde hsrt af hans Mund . V . 20. Vistnok maastee med det hemmelige Snste , ikke at komme ncermere ind paa en for hende ydmygende og befioemmend e Materie , men visselig en ogsaa dreven af en oprigtig og hjertelig Attraae efter en for hendes Samvittighed vigtig Oplysning , foreloegger <b>Qvinden</b> ham , i hvem hun erkjendte en Prophet , d . e . en af Gud om guddommelige Ting oplyst Mand , en Sag , der rimeligviis oftere havde bestjoeftiget hendes Tanker , idet hun onsker at erfare hans Mening om , paa hvilket Sted Gud rettest , eller meest overensstemmende med hans Villie , tilbades , enten , som Samaritanerne gjorde , paa det ved Sichar liggende Bjerg Garizim , eller , efter Zooernes Paastand , i Jerusalems Tempel . Paa Garizim , det Bjerg , fra hvilket Mose og siden losva forkyndte Velsignelsen over Folket ( ligesom Forbandelsen fra det lidt nordligere liggende Ebal , 6 Mos . 11 , 29. ; 27 , 12. 13. losv . 8 , 33. 34. ) , havde Samaritanerne , paa Alexander den Stores Tider og med hans Tilladelse , bygget et Tempel , hvor de holdt deres Gudstjeneste . Dette Tempel forstyrredes af Makkabceeren Johannes Hyrka-

<h4>2232</h4>35. 2 - 4. Istedetfor ligefrem at besvare en Anklage , der aldeles ingen Grund havde at stotte sig til , og hvis Falsthed han vidste , at Dommerne selv kjendte , aflægger Stephanus , dreven af Guds <b>Aand</b> , et Vidnesbyrd om det gjenstridige Israels Forhold til Gud , der fuldkommen tilstrækkelig blotter hans Modstanderes Brode og godtgjor hans egen Uskyldighed , hans dybe Mrefrygt for Fcrdrenes Gud ( 35.2 - 19. ) , for Moses ( 35.20 - 37. ) , for Loven ( 35.38 . ) , og for Templet ( 35. 45 - 47. ) , ved Siden af Modstandernes Foragt for samme . Han gaaer tilbage i Tiden til Aabenbarelsen for Stammefcedrene af den samme Naade og Miskundhed , der lyste alt klarere og klarere gjennem Tidernes Lob for Israels Folk , indtil den i sin fulde Herlighed , med alle Forjættelsernes Ja og Amen , straalede i Jesu Christi hellige Person og dyre Forlosningsvcrrk. Og han viser derhos , hvorlunde det stedte , at , alt som Guds Magt og Kjoerlighed klarere aabenbaredes , saa kundgjorde sig ogsaa Modstanden derimod af de Vantroe og Übodfcerdige , indtil denne Modstand fyldte sit <b>Maal</b> i Jesu Mordere . Fra Abrahams Tid begyndte de tydelige Spor at <b>vise</b> sig af Guds Raad med Hensyn til Israels Folk . " Herlighedens Gud , " den Gud , i hvem al Herlighed , al Majestcet , al Fuldkommenheds Fylde boer , den evig Priisvcerdige , « åbenbarede sig , gav sig tilkjende for hiin sin Udvalgte . Abraham var fsdt i Ur-Kasdim ( d . e . Chalotternes Ur ) , 1 Mos . 11,28 . Denne By laa i Mesopotamien ( 2 , 9. ) i udstrakt Betydning , nemlig i Landets nordlige Deel , der beboedes af Chaldceere . Herfra drog Faderen Tha rah med Abraham og hans Hustru

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008112003005'target=_>Munch, P.A., 1857, Det norske Folks Historie</a><h4>3497</h4>traadte frem imellem dem , og talte saaledes ! „ Vel er jeg ikke synderleg vant til at tale paa Thinge eller i store Forsamlinger , men det er en serdeles vigtig Sag , som nu driver mig til at trede frem her , og jeg onstrr at jeg maatte fremsette den saaledes , at I alle kunne forståa og begribe hvad jeg mener . Jeg er kommen til <b>vis</b> <b>Kundskab</b> om , at min Broder Haakon Jarl har sogt at overtale mange Hovdinger og haandgangnr Mend her i Landet til at give ham Kongenavn , og at mange virkelig have givet ham opmuntrende Svar , sigende som sandt er , at han ikle er mere uskikket dertil , end jeg . Han lader sig ogsaa forlyde mrd , at jeg skulde have ytret noget om , at jeg ikke i mindste Maade vilde lade det mig fortryde , om han , min Broder , kaldes Konge lige saavel som jeg over mit Nige , siden jeg sinder mig iat Philip kaldes Konge . Men det vide jo dog alle , at min Broder og jeg have tåget Kongenavnet fra Kong Philip ved Hvitingso , og at han derpaa har maattct aflegge os Gd , hvorved det mere bliver kun til Spot end Wre , om han nu druger dette Navn . Jeg derimod har faaet Kongenavn paa Orething , ug alt Landets Folk har svoret mig Trostab og Lydighet » , lige saavel Haakon Jarl selv og Philip som andre , jeg har derimod itte aflagt Ed paa andet end at holde Indbyggerne ved Lov og Net . Derfor skulle I alle vide , at saa lenge jeg lever , vil jeg ikke vide af , at nogen anden end jeg stal kaldes Konge i Norge , dersom I kun ville holde eders Ed og staa mig bi . " Hertil svarede alle vel , berommede hans Tale , og erklcerede eenstemmigt , at de ej vilde erkjende nogen anden Konge end ham , ja de takkede ham endog for hans Ord og sagde at han talte bedre end de havde troet ham i Stand til . Haakon Jarl maatte saaledes for det forste opgive disse hemmelige Anslag om at skaffe sig Kongeverdigheden i Kong Inges levende Live , men fandt paa en ny Udvej , idet han mrd sine Venners Bistand , og aldeles aabenbart , fordrede at Inge stulde indgaa en Overeenskomst med ham om , hvorledes det skulde forholdes , nåar en af dem faldt fra . Og heri lyttedes det ham ogsaa at faa Erkebiskoppen og de ovrige Vistopper paa sin Side . Forst ankede man over at Inge ej var egtefod , fordi hans <b>Moder</b> Cecilia, som det hed , giftede sig med Baard Guthormsson medens hendes forrige Mand endnu tevede , og Vistopperne fornyede med et den ved Modet i 4 M vedtagne Bestemmelse , at den , der efter Loven stulde vere Konge , ej alene maatte vere af kongelig Vyrd , men ogsaa egtefod . Kong Inge oplyste imidlertid , hvad der forhen er omtalt ( S . 196 ) , at hans <b>Moder</b> havde egtrt Baard med Erkebiskop Eysteins Tilladrlse , og at hendes foregaaende saakaldte Egteskab med Follvid Lagmand kun var Konkubinat , saa at det altsaa var Haakon , itte Inge , som var uegte fod . Da det saaledes ikke nyttede at angribe Inges egen Vyrd , rettede man Angrebet mod hans Son . Han havde nemlig en uegte Son , ved Navn Guthorm , som en

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111103010'target=_>Hansen, Maurits, 1857, Mauritz Hansens Noveller og Fortællinger</a><h4>2056</h4> la , siden vi skrive 1830 i Aar , og jeg gik op 1820. — Altsaa , da jeg var fcrrdig med min Eramen , sik jeg det Indfald at beftge Danmark, hvor jeg har nogen Familie . Jeg logerede mig < nd hos en Madame Homand i Frederiksberg , en brav , ferm <b>Kone</b> . Det faldt mindre kostbart , end inde i Byen , og jeg havde lidet at rejse med. Den sterste Del af Dagen tilbragte jeg med at bese Kjobenhavns Mcerkvcerdigheder og besoge mine Vestcrgtede og Vekjendte ; Theen om Morgenen og Smerrebrodet om Aftenen med en Flaske Pindsl til tog jeg hos og som oftest i Selskab med min aldrende Vertinde , som da fortalte om Frederiks Hospital , hvor hun i mange Aar havde vceret Vaagekone og af gamle Vang havde fortjent sig mangt et :  Hun er for den Skam en skikkelig <b>Kone</b> , Madame Homand . Vor Herre vil vist hjelpe hende ; for hun tjener ham paa den bedste Maade : hun gjsr sin Gjerning tro " . <b>Mor</b> Homand snakkede gjerne om Vang , men endnu heller om sin salig Mand , Skibstommermand Homand , og aller helst om sin Datter , der var i Huset hos en Modehandlerinde i Kongensgade ,  en annerleens <b>Kone</b> , end andre Modehandlerinder . Hun havde Lovise som sit eget Barn , og gav hende al den <b>Lcrrdom</b> , den fornemmeste Mands Datter kunde onske sig " . — Denne Berpmmelse gjorde , som Du kan begribe , ikke sunderligt Indtryk paa mit Sind ; men da Lovise en Sondags Eftermiddag uventet kom hjem , idet jeg fad ved Kaffebordet med min Vertinde, og jeg forste Gang saa den blomstrende Jomfru , Horte hendes melodisie Stemme og deklamatorisie Foredrag : da felte jeg en <b>vis</b> Varme om Hjertet . Dog , der siulde mere til . Et Trcek af Louises ualmindelige Sjcrl bestemte min vaklende Beslutning . Ikke hun selv , ikke Moderen, men en Nabokone fortalte mig en Aften , da min Vertlnde var ude , dette Trcrk fra Louises tolvte Aar , saaledes , som jeg nu vil gjentage

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012101624040'target=_>Holberg, Ludvig, 1869, Holbergs Comedier</a><h4>7709</h4>Det er underligt Slags Folk disse Poeter . Om en anden wrlig Mand giver et Nrefigen udi Tanker , trcekker man ham til Daare - Kisten ; men hos dem er saadant Tegn til <b>Lcrrdom</b> og Dygtighed . Ellers fik jeg det Vers for got Kisb . De Folk maa have nogen Frihed , efterdi de ere saa lidet interesserede ; thi hvad er dog 12 tz . for saadan lcrrd Mand ? Jeg forstaaer ikke Verset , men kand dog see , at det er altiid sine Penge vcrrd . Nu stal jeg bilde min Hosbond ind , at jeg har givet 2 Rirdlr . derfor , saa vinder jeg ved dette ZErende 11 Mark og 4 h . Jeg haver kun liden <b>vis</b> Lsn , maa derfor neere mig redelig ved Commissioner . Mangen brav Mand fsder <b>Kone</b> og Bsrn derved udi Kisbenhavn ; thi hvoraf skulde de gode Folk ellers leve ? Hvis jeg kunde flattere , som Jesper gisr for min Herre , kunde jeg have de Smautzer og de Forceringer , som han faaer . Jeg skulde ikke roese mig selv ; men jeg er en cerlig Karl , der ikke land sige andet , end jeg mener . Naar min Hosbond begynder at tale om sin Manddom i den Brabandsie Kriig , tier Peer gandste still ; thi jeg veed , han har aldrig vceret paa den anden Side af Belt , end sige udi Brabant , som ligger nogle tusinde Mile herfra . Ney JErlighed varer dog lcrngst ; det heder hos mig , som

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111500033'target=_>, 1863, Bibelen, eller Den hellige Skrift</a><h4>1285</h4>Dit Lys bli ' r klart din Stjerne opgaar . " Noget af dette har allerede enhver Sjcel , i hvem Zions <b>Aand</b> lever , erfaret i Naadens Tider , det er en Forsmag , hvorved det bliver os muligt at forståa det profetiske Ord . Dit Lys er kommen og Herrens Herlighed er opgangen over dig . Sml . Es . 9 , 2.  Dit Lys " det Lys , hvis Glands skal oplyse dit Morke , din Guds Lys , Herrens Herlighed opgaar over dig . Det er  Lyset fra det Hoie , " hvorom Zacharias forkynder Lue . i , 78 — 79. Jesus i Tjenerskikkelsen var allerede dette Lys , mm i hans anden Aabenbarelse skal det vcere endnu herligere endnu niere utilhyllet. Hans fsrste Komme i Kjodet var Tegn og Forbilledc paa hans andet Komme i Herlighed . Sml . Ap . Gjem . 1 , 11. — Og Hedningerne skulle vandre ved dit Lys . Sml . Es . 2 , 2 — 3. Hvor der er et Aadsel . der samle Ornene sig ; hvor der er et Lys didhen samle Lysets Born sig fjernt og ncrr . — Og dine Dstre opfostres ved Siden . Rigtigere saaledes som der staar Es . 66 , 12 : dine Dotre  sknlle boere s ved Siden . " At boere ved Siden er efter hebraisk Sprogbrug det samme som vort : at bcere paa Armene . Dine Dotre skulle af Me <b>Modre</b> og Ammer bceres til dig paa Armene , ligesom Bornene Mare . 10 , 13. bleve baarne til lesum . Sml . Es . 49 , 22. Saa stor stal Iveren vcere , at Ingen bliver tilbage , men Alle komme til Zion . Disse Zions Sonner og Dotre ere ikke blot legemlige Eftertommere af Abraham , men ogsaa aandelige Abrahamsborn . der ere af hedensk Wt , men formedelst Troen ere fodte af Aanden . Sml . Es . 44 , 5. Ps . 87 , 4 — 6. Rom . 4 , 9 - 18. - Og dit Hjerte stal rcrddes og udbrede sig , naar Mangfoldigheden ved Havet omvendes til dig .  Rceddes " eg :  bceve " nemlig af glad Bevcegelse og udvides saf Glcede ; Frygt og Angest derimod gjsre Hjertet beklemt). Mangfoldigheden ved Havet noevnes forst , fordi Landene ved Havet sDerne ) i Vesten betegne de fjerneste Egne og at disses Beboere komme , vcekker den storste Forundring og Glcede . Under den videre Udforelse af denne Skildring stilles dog Beboerne af Osterlandene, der komme paa Kameler , foran som de , der ere ncermest sB . 6 — 7 ) ; derefter folge Vesterlandenes fjernere Beboere , der komme paa Havskibe sV . B — 9 ) . — Veilsbere af Midian og Efa . Veilsbere o : Kamelfol , unge raske Kameler , der lobe hurtigere end de gamle . Alle de af Scheba skulle komme . I Midian Efa og Saba sSeba ) boede legemlige Efterkommere af Abraham ved hans anden <b>Hustru</b> Ketura , som han cegtede efter Såras Dod sl Mos . 25 , I — 4 ) . Ogsaa i legemlig Henseende vare disse saaledes allerede Abrahams Born . Som Erempel , og uden at de andre Broderstammer udelukkes , blive de derfor her forst ncevnte som saadanne , der ved Troen ogsaa stulle blive Medarvinger til Israels Forjettelser . Fra Saba kom ogsaa engang Dronningen fra Arabiens Rige med Kameler , Guld og Specerier ( Urter ) , hvoriblandt ogsaa Rogelse , for at hore Salomos <b>Visdom</b> og se hans Gjerninger li Kong . 10 , 1 — 10 ) . Men nu stulle Alle fra Saba komme for at se og hore . Her er mere end Salomo ! Sml . Pf . 72 , 10 - 11. De skulle berre Guld og Rsgelse osv. Begivenheden i Matth . 2 , 1 — 11 er ikke endnn den endelige Opfyldelse af denne Spaadom , men vel et Tegn paa . at Jesus .  den nyfodte lodernes Konge , " var den , ved hvem tilsidst Gnds Herlighed stulde

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016103148541'target=_>Berg, Ole, 1863, Prædikener over Evangelietexterne paa Kirkeaarets Søn- og Helligdage</a><h4>229</h4>Alle de , der nærmere kjendte Sognepræst Berg , ville have faaet Indtrykket af , at han var i Besiddelse af usædvanlige Aandscvner og gode <b>Kundskaber</b> , at han navnlig var en sjelden skarp Tamfer , der med Klarhed , Liv og Lethed , i Forening med en <b>vis</b> eiendommelig Humor , førstod at give stue Tanker Udtryk . At dette hans skarpe kritiske Blik i Forening med hans humoristiske <b>Begavelse</b> ofte gjorde ham noget haard og hvas i sine Domme , staner ikke til at nægte , men de , som i hans sidste Aar kjendte ham nærmere , ville ogsaa vide , at han paa Bnnocn bar et Gemyt , der var boiet i Aandens Skole og gjennemtrccngt af christelig Forsonlighed . Han var en tro og kraftig Arbeider i sin Herres Tjeneste ; sine Kaldspligter rogtcde han med Nidkjccrhed og stor Uegennyttighed; hans Virksomhed i sin Menighed bar ogsaa oiensynlig rige Frugter ; i Erkjendelse heraf har Menigheden besluttet at reise et Minde over hans Grav og at anskaffe et Portratt af ham til at hcenge i Kirken . Han var dcrnæst en kjcrrlig og om VEgtefcelle , der „ vel forestod sit eget Huus ; " den Maade , hvorpaa han omgikkes sin blide og fromme <b>Hustru</b> , ved hvem han hang med den inderligste Kjærlighed, mindede om Pauli Ord : „ forekommer hverandre med Wrbsdighed!" For sine Smaae , hvoraf de 6 — 4 Dottrc og 2 Sonner — endnu ere ilive , — 2 vare gcmgne forud , — var han en kjærlig Fader og en christelig Opdrager . ' Han var ogsaa en trofast Ven og er som saadan savnet og erindret af Mange .

<h4>427</h4>ogsaa paa dette Ords utvetydige Vidnesbyrd om hine høiere Aanders Deeltagelse og Indgriben i denne synlige Verdens Begivenheder , maa det fremstille sig som ncitnrligt og rimeligt nok , at en himmelsk Hærstares Mangfoldighed jublede ved hans Fodsel , som er Menneskelivets Sol og den ganske Verdenshistories straalende Midtpunkt . Om Englene , der de lovede Gud og sagde : „ 3 Ere være Gud i det Hoieste ! " selv klart gjen«emfkuede den hele Forløsningsvcerkets Hemmelighed , hvori de , som Skriften vidner , begjærede at indkige - det vide vi ikke ; thi derom er Intet os aabenbaret ; men Saameget kunne vi af vort Evangelium klarligen see , at de vidste , at det var en Frelser , som var fod til Verden , og at de , idet de kundgjorde for Hyrderne Budskabet derom , forkyndte en stor Glade , som skulde vederfares alt Folket . Der kan ingen Tvivl være om , at de jo vidste , at han var den allerede strar efter Syndefaldet forjættede <b>Kvindens</b> Sced , som skulde knuse Slangens Hoved , den Abrahams LEtling , i hvem alle Jordens Slcegter skulde velsignes , den Fredsfyrste , som stnlde komme i Tidens Fylde , hvis Udgang var fra fordum , fra Evigheds Dage . De vidste nok ogsaa , at han var ingen Ringere end det evige Ord , som var i Begyndelsen hos Gud , og som selv var Gud , og som nu var bleveu Kjød og havde taget Bolig blandt Menneskene , — at han var Herlighedens Afglands og hans Væscns ndtrykte Billede , aabenbaret paa Jorden for at forherlige Faderen og for at deelagtiggjore Menneskene , hvis Broder han var vorden , i sin egen Herlighed . Derfor kvad de : „ 3 Ere være Gud i det Hoieste ! " Og vi gjenloste , dyrt kjøbte Christne ! skulle vi ikke istemme med i dette Kvad og sige : Ære være Gud i det Hoieste ? Selv de iblandt os , som endnu ikke , oplyste ved Guds <b>Aand</b> , have skuet Jesu Herlighed som den Eenbaarnes fra Faderen fuld af Naade og Sandhed , kunne dog ikke , saasandt de ikke ere aldeles blinde for de store Kjendsgjerninger , som Historiens Aarboger og Erfaringens Vidnesbyrd klarligen lægge for Dagen, miskjende eller nægte , at lesn Christi Indtræden i Verden har ikke blot i Eet og Andet men i alt Væsentligt og heelt igjennem omdannet det hele Menneskeliv og bestemt Verdenshistoriens Gang . Her er ikke Sted til yderligere at paavise dette ; det behoves visseligen heller ikke ; Enhver seer det og maa erkjende det , som kjender en Smule til , hvad der har tildraget sig og tildrager sig paa Jorden . Jeg vil kun pege paa den Kjendsgjerning : overalt paa Jorden , hvor Menneskene have skredet frem til nogen høiere Grad af <b>Aandsdannelse</b> og høiere Udvikling , der herster Christendommen , og overalt , hvor 3 *

<h4>763</h4>Maal giver Alle og Enhver Anledning , men hvortil , ak ! saa Faa knnne beslutte sig , fordi Kjødet saa nodig vil underkaste sig den haarde , men saa saare vclgjorende Tugt , som oves i den Skole , og selv iblandt de Faa , som beslutte sig til at underkaste sig denne Tugt , er der uok af dem , som blive tratte , for <b>Maalct</b> uaaes , efterdi Aanden vel var villig , men Kjødet viste sig at være sirobeligt . Deraf kommer det , at de Fleste — ja , hvor frygter jeg for , at jeg maa sige de Fleste ! — nok snarere berande sig med alt Andet i denne hoivigtige Sag end med Gno i Himmelen , som de saa lidet kjende og saa lidet ere vante til at beraade sig med i vigtige Anliggender . Nogle spørge Kjød og Blod tilraads , Andre see hen til timeligt Gods eller verdslig Hader , Andre tage aldeles intet Hensyn , men fare afsted i Letsindighed og Overilelse eller tilskyndede af et flygtigt Luue eller Indfald først til Jammer og Moie og Elendighed og Sorg og Anger for , efterår have gjort Livet bittert for sig selv , at forbitre det ogsaa for Andre og navnligen for dem , hvem de have forpligtet sig til at være en Stotte og Medhjalp , og siden , uaar Livets vilde Leg er endt , — ja hvorhen fare de saa afsted ? Ak ! vi vide jo nok , hvorhen de nødvendigen maae fare afsted , som ikke kjende Gud og den , han ndscndte , lesum Christum , og som derfor ikke sporge Faderen tilraads i den eenbaarnc Sons Navn om deres Livs Veie og om de Skridt , de gjore paa den farlige Bane . — Men lader os videre undersoge, hvad der sporges om i hint første Sporgsmaal af Mgtevielsesformularen. De Trolovede sporges fremdeles , om de have bcraadt sig med deres , eget Hjertelag . Dette mene de vel som oftest at have gjort ; men ak ! hvor daarligt have de ikke mangen Gang beraadt sig med deres eget Hjertelag . Her forudsattes uaturligviis , som overalt i Ritualets og Alterbogens Tiltale til Menigheden eller deus enkelte Lemmer , at det er Christne Talen rettes til . Hjertelaget skulde altsaa efter Ordenes Mening vare en Christens Hjertelag . Meningen af dette Sporgsmaal er altsaa aabenbarligen denne : „ Har du , Jesu Christi Tjener eller <b>Tjenerinde</b> , taget dit Hjerte , hvori jo , saasandt du er i Christo Jesu , den Helligaaud har fastet 80 , med paa Raad , idet du valgte din Livs Ledsager eller Ledsagerske ? Har du fuudet Jesu Christi Sind i den , til hvem du vil slutte dig for Livet ? Er du forenet med den , dit Hjerte begjarede , i Aaudeus Samfund , i Jesu Christi Tro og i Faderens Kjarlighed? Er du <b>vis</b> paa , at han , som dn af ganske Hjerte der og cerer og dyrker som din Fader , som er i Himlene , ogsaa er dens Fader , som du skal kalde diu ZEgtefalle , og med hvem du stal dele Livets Glader og Livets Sorger , Livets Arbeide

<h4>2182</h4>robe sin Sorg for sig selv ; thi den Sorg , jeg mener , er af den Beskaffenhed , at man helst vil glemme den , for at ikke Livet ved den stal aldeles forbitres . ' I ane maastee , mine kjære Tilhorcre ! hvad det er for en Sorg , jeg sigter til . Det er nemlig netop den , som Dagens Evangelium noder os til at tænke paa , den Sorg , som den grædende <b>Moder</b> og Sorgestaren og Ligbaaren stiller os saa levende for Die . For den Sorg er selv den Rigeste ikke fri ; den nager Kongernes Hjerter paa deres Throne , og ingen menneskelig Magt kan bortrydde den ester endog kun mildne den . Den , hvis <b>Maal</b> er at samle Gods og Guld , har foruden alle de andre Sorger og Bekymringer, som den Afgud , han tjener , volder ham , altid ogsaa denne : „ hvorlænge vil dette vare ? og naar min Tid er omme , hvad saa ? " — og en sand Folelse , som han ikke aldeles kan fornægte , siger ham , at hans Skatte ikke kunue baade ham , naar haus Legeme smuldrer i Graven , og en ikke mindre sand og paalidelig Stemme i hans Indre maner om , at det , selv om hans Tanker kunde førsone sig med den for ethvert folende Menneskehjerte saa forfærdelige Forestilling om fuldkommen Tilintetgørelse og sporlos Forsvinden ved Døden , iAe engang er ham muligt for Alvor at fastholde denne Forestilling , hvor sorgelig og utilfredsstillende den end i ethvert Tilfalde vilde være , men at han uodes til at tceuke sig som tilværende efter Døden , og hvad kan da det samlede Gods baade de « fattige og uogne Sjcrl , som ingen andre Skatte har end de , som maatte lades tilbage paa Jorden ? Og den , som lever for at nvde Livet , maa dog sikkert nu og da foruroliges af den Tanke : „ der kommer en Tid , da du ikke kan nyde mere det , som nu er din hoieste Glæde ; " ja , i Grundeu kaster vel en mere eller mindre klar <b>Bevidsthed</b> om denne sorgelige Sandhed stadigen en msrk Slagstygge om alle hans Livsnydelser . Og den , som udeu at omtumles af vilde Lidenskaber og uden at foruroliges og ængstes af Sorger for dette Livs Næring , lever et roligt og behageligt Liv , nydende det huuslige Livs stille Glæder ien elsket Families Skjod , ogsaa en Saadan maa jo stundom gribes af den ængstende Tanke : „ hvorledes er min Lykkes Borg grundet ? kan den trodse Verdenslivets Storme og de verlende Skjcebners ofte saa grusomme Spil ? " I Dageus Evangelium fortælles der jo om en <b>Kvinde</b> , der vel engang havde været lykkelig som <b>Hustru</b> og <b>Moder</b> , men nu var hun Enke og hendes Ssn , den eneste , laa paa Ligbaaren ; og slige Tilskikkelser ere jo ingenlunde sjeldne i Livet . „ Hvad om noget Saadcmt hcendte mig ? hvad om Døden trceugte ind ogsaa i min Kreds og bortrev mine Kjære , En af dem eller Flere af dem , eller , o Gud ! dem

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011050308005'target=_>Wexels, W.A., 1846, Christelig Huus-Postille, eller Betragtninger til hver Søn- og Hellig-Dag i Aaret</a><h4>368</h4>aabcnbarcr indtil Dagcncs Ende dc Tanker , det Hovmodens og Egcnkj-crlighedcns Sind , der ofte dybt stjult under en mcnncstclia Elffvcrrdiahed og Retsiancnhcd og hvcrkcn ftcs af Mennesket stlv , i hvis Hjerte det bocr , eller af ftrend Christus <b>moder</b> Hjertet ; og han <b>moder</b> Alle — Ingen kan cvindcttg undgaae Sammcnsiodct med barn — d ? t vccrc nu lil Fald eller til Oprcisnina. Salig Den , for bvem han bliver det Sidste ! Endnu Ekikkelft fremtrcrdcr der l den Scene , Dagens Evangclmm forcholdcr os . Det var Anna, Phanucls Dattcr , en med prophcttsk <b>Aand</b> begavct , fire oa Ovindc , der . gamle Sl- tjcntc Guo i Oprigtighcd paa det Standpunkt , hun slo ? , men ventcoe sin Salighcd af hans frie Naade i « dam , der skulde komme » . " Hun traadte til i samme Stund oa priscde liacledes den Herre , der besogte sit Folk , crkjendtc Icsus for Christus og talcde om Guds Naade i ham lil alle de Fromme i Staden , som ventede paa Forlosercn . Hvor glad har itte ogsaa hun v < rret og hvUket Lys bar hun itte sprcdt omkring sig ! Ringe og overftet af Vnden , eicdc oa mcddcelte hun hvad ikke Verdcn i dens hoieste Viisdom / i dcns hcrligste Pragt , kjender , og da hcndcs Time kom , bcnsov Fred , og gik ind med Simcon til Hvilcn i Abrabams til glcedelig For- mcd dam , deres ei blot kjodclige , men aandcligc Etammcfader , der med Begjcrrlighcd havde stuet den Dag imode i Aandcn , som hine hans tvcnde Vom nu havde stet i Kjodct ; og idet Guds Finger peger for os paa de tvcnde Vovwandrcnde , siger ban : Scc der , ne Aars 2 lftcn skal blive blid ! Sce der , hvorlcdcs dit Fan vel til Livcts svundnc Dage sia ! blive mcd Glcrde ! la , christne Venner ! en blid Livets Aften , et glced cligt Farvel til alle vore Dage paa lorden , naar deres <b>Maal</b> er fuldt — er det ikke dc t , vi niaae onsse , saa ofte alt det lordisscs Forkra , ' nt ' c ! ighed , Dodcns Vished og vor eviac Skjcrbne staacr os for Die ? Onftc vi dog alt lercde , at bvcr enkelt Dag , efter dens bestlkkcde Deel af ' Arbcioe , Sorg og maa ende sig blidt — hvor me »

<h4>657</h4>den onde Dag ; ja , naar vi hore Apostelen efter sin himmelste Henrykkelse sige , at han af saadanne Ting kun vilde rose sig , forsaavidt de vare Vidnesbyrd om Guds Naade , at denne Naade var ham Nok , at han , hvad ham selv angik , kun var glad i den i hvilkcn Herren fuldkommede sin Kraft , o , da onste vi os ikke at blive henrykkede med ham i den tredie Himmel , men hige kun efter at kunne , som han , vandre i Troen og blive Vidner om den guddommelige Naadcs og Krafts Virksomhed i et strpbcligt Kar ; og , naar vi hore ham fon kynde , hvorledes svare Lidelser maatte tilstode ham , for at han ei skulde hovmodes af sine Henrykkelser , sine hoie Nydelser , da fatte vi , at , savne vi disse , da er ogsaa dc t godt , at , vandre vi mere i Troen , end i Folelsen , mere i en stille Lengsel , end i nogen scerdelcs <b>aandelig</b> Glcedes Fornemmclse , mere i Haabet end i Nydelsen , da staae vi heller ikke i Fare for at tillcegge os en Fuldkommenhed , vi ei besidde , at drsmme om en <b>aandelig</b> Storhed , der ei er vor , at see med Ringeagt eller stolt Medlidcnhed ned paa dem , der ikke nicerke nogen scerdcles og paafaldende Hendragclse i den overjordiste Vcrden , da bcvares vi desto lettere i Idmyghed , i Selvfornedrclse , i den Eenfoldighcd til Christum , som er meget kostelig for Gud . Hvad nu den anden De el af Terten angaaer , om den Pcel i Kjsdet , som var givet Paulus , for at ncdknuge al Hovmod hos ham , for at afholde ham fra at ophpie sig af de store Aabenbarelscr , der vare ham givne og om « den Engel Satan , der slog ham med Nawe , » plagede , piinte ham haardt . og fra hvis Angreb han ikke paa sine Bonner blev befriet , da <b>moder</b> os ogsaa her Dunkelhed; thi den ncermere Vcstaffenhed af Pauli Lidelse kunne vi ikke afgjsre ; men det er heller ikke npovendigt ; at det var en tung , smertende og trykkende Lidelse , ikke alcne for Legemet , men ogsaa for Aandcn , en Lidelse , der ret var stikket ti ! at ydmyge Hjertet pg bringe hver Hovmods-Tanke om <b>aandelig</b> Fuldkommenhed til at tie , det kunne vi ssjonne , og det er os Nok til at kunne uddrage gavnlig <b>Lcerdom</b> deraf , til at takke den <b>Aand</b> , som

<h4>779</h4>tes ind i Morket , ind i dybeste Smerte , men gik gjennem Trcengselen uden at flippe Ordet af deres Hjerter , og greb omsider , hvad de higede efter , greb Klenodiet , som Natten indhylledc . ' Visseligen , det vilde vcere et qvcegende , et styrtende Syn . Medchristne ! vi have et Ord , endnu herligere , endnu klarere , endnu rigere og mcegtigere end det , hiin <b>Qvinde</b> havde hort ; vi have Ordet om Kjcerligheden, der blodte for vore Synder paa Forbandelscns Trce , om Kjcerligheden , der dsde , men tog Livet igjen , for at dele det med os evindelig . Dette Ord , med hvcrt af de ovrige , som dertil knytte sig , om denne Kjcerligheds Villighed til , ja Lcengsel efter at give os Alt , hvad vi trconge til , Alt , hvad der i Sandhed fremmer vort Vel , om denne Kjcerligheds uendelige Omhu for vor <b>Sjel</b> , det stulle vi holde fast ved , naar vi Intet sec , Intct fole , uden Synd og Dsd og Sorg ; med det i Hjerte og Mund stulle vi bedendc kcempe , saalcenge indtil vi hore Herrens Ia og Amen ; det stulle vi foreholde ham , hvergang han synes at ville trcekke sin Haand fra os , at ville , vi stulle gaae hen i vor Jammer uden Hjcelp ; med det stulle vi nedslaae alle morke Tanker , alle crngstende Tvivl , alle Roster , som ville gjore vor Vsn , vort til en Daarlighed , om saadan Nyst end tyktes os at komme fra Herren selv ; vi stulle omklamre dette Ord med alle <b>Sjelens</b> Krcefter og ikke lade det vriste fra os paa nogen Maade , — og da kommer ganste vist re ns Time ; hvor svag din Tro kan tykkes dig , den stal dog overvinde Herren , den stal dog triumpherende hjembcrre fra Kampen , ikke alcne den Naade , den sogte , men uendelig meget Mere , en storre Gave og Rigdom end den nogensinde kunde ane , ja den stal hjembccre , ikke alene Hundenes Smuler , men Vornenes Brod i fuloestc <b>Maal</b> , og den stal omsider hjembcere Livets Krone .

<h4>1090</h4>lige Fortolling , som vi i dens Udftrlighed lcese i Apost . Gjem . det 10 ende Capitel , Anlcdning til , med Von om Guds naadige Bistand , at betragte : Den ced leste Lcengsels eneste og sikkre Tilfredsstillelse . Det er en sanddru og gribende Skildring af Mennestets Tilstand og Kaar , som vi lcese hos den <b>vise</b> Sy ' rach i hans Vogs 4 de Cap . 1 ff . Vers , hvor det blandt Andet hedder : » Stor Urolighed er stabt for hvcrt Menneske, og der er et tungt Aag paa Adams Born fra den Mag , de udkommc af deres Moders Liv , indtil den Dag , Ue komme til Alles <b>Moder</b> igjen . I deres Vetcenkninger M Hjertens Frygt er der en cengstelig Forventning , fra ' Den , som sidder paa 3 Ercns Throne , og indtil Den , som er fornedret til Jord og Asse , fra Den , som bcercr Silke og Krone , til Den , som er klcedt i grovt Lcerred . Der er Forstyrrclse og Bevcrgelse og liden eller ingen Hvile , « la , Venner , der er en Uro , en dyd Uro , i Mennestets Hjerte , og den er ikke borte , fordi Overstaden stundom er smilende , og Drommcn om « Fred og ingen Fare « sod ; der er en bevceget Lcengsel , en Altraa , en S ? gen , en Stunden efter Noget , thi der brcender et Savn derinde. Hvad Mennesket savner , hvad det lcengcs efter , er det ofte selv dunkelt ; men — det lcenges , og med denne Lcengsel gaacr Mcnncstet ud og soger Hvile , og den gribcr da , hvad der <b>moder</b> i Livet og lover denne Hvile , den folger da , hvor Hjertet drages hen , den slutter sig da til og nedscenker sig i Det , hvoraf Hjertet tiltales. Efter Bestaffenheden heraf , efter Bcstaffenheden af det <b>Maal</b> , Lcengstlen har for Vie , bliver denne Lcengsel i storre eller mindre Grad uced el eller ced el . Ucedel bliver den , naar den gaacr ud vaa det Forbudne , paa Det , hvorom Guds Lov siger : du stal ikke begjcere det ! — det vcere nu ureent i sig selv eller Noget , som ikke stal gives , ikke stal tilhore os , det have nu sit Hjem i Aandens eller i Legemets Vcrden ; ucedcl er ogsaa Lengselen, <b>Sjelens</b> Lccngsrl , naar den kun gaacr ud paa Det , som er Stov og Aste , paa Det , der deler Forkrcen-13"

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111303026'target=_>Rudelbach, Andreas Gottlob, 1848, Christelig Biographie</a><h4>193</h4>et Menneske i denne Tid ncermcst monne komme til Fllldkom,ucnhedens <b>Maal</b> . Mesteren undsiog sig derfor med den Vemcrrkning, det var ikke saadaune hoie Ting , man prcedikede for Lcrgfolk om ; men da Manden svarede , han . fsltc dog saa inderlig Lcrngsel dcrefter , og det var vist en salig Frugt af Arbeidet , naar blot een af alle de Sjcle , som nu lobe efter hans Prcedikeu , forstod ham , saa lovede Mesteren at opfyld ? hans Vegjering , naar han forst havde sinderet ret derpaa . Da nn Dagen kom , som han bavde bekjcndtgjort for Folket , holdt han en kostelig Prcediken on : de fireogtpve Stykker , der hore til en sand <b>aandelig</b> Oplysning, saa Christus monne sige om et Menneste :  See , det er en sand Israelit , i hvilken intet Falsk er " ( Joh . 4 , 47 ) ; og vilde han , som han sagde , afholde sig fra , hvad der ellers var hans Vane , at indblande og bevise Alt , hvad han udfortc , af den hellige Skrift . Da Manden nn havde hort denne Prcrdiken , ^ gik han til sit Herberge og ovskrcv den Ord for andet . Derpaa gik han til Mesteren og bad ham om Forlov , at maatte oplcese den for ham . Mesteren blcv hoist forundret over , hvor sindrig han havde opfattet og beholdt Alt , og da Manden lod til at ville vende hjemad , bad han ham blive , siden han dog hverken havde <b>Kone</b> eller Born hjemme og fortcerede ligesaa godt der , som her . la , kjcrc Mester , suarede denne , I stal vide , det var ikke derfor jeg kom hid , for Eders Prcedikens Skpld , men for med Guds Hjelp at skaffe Raad paa en . eller anden Maade . Hvad Naad vilde " du skaffe , sagde Mesteren , da du dog er en Lccgmand og ikke forstaaer Skriften , og ei har nogen Nel til at prcedike ? Bliv her lcengere ; maaskee giver Gud mig at holde dig endnu een Prccdiken , saadan en , som du gjerne horer . Men Manden sputes at have noget Andet paa Hjertet ; han bad atter om Forlov at tale , og sagde nu omtrent Fslgcnde til Mesteren :  Vistnok er I en dygtig Prcrst og har i denne Prcrdiken givet en god <b>Lcrrdom</b> ; men I leve ikke selv derefter . I har dog selv ftrcediket , at , naar den hoieste Mester i al Sandhed kommer til Mennesket , da maa denne affore sig alle forgjengelige Ting . Viid , uaar denne Mcstcr kommer til mig , da lcrrer han mig mere i een Time , end nogen Lcercr i al Evighcd kan . " Da nu Mesteren bad Loegmandeu paa det indstcendigstc , at han dog skulde bli ^ , og denne

<h4>1088</h4>Legemet , dog ikke til Geilhed " ( Rom . 13 , 11 — 14 ) . Og jo mere udbrcdt og levende Kampen mod Hedenstabet blev , desto stcerkcre klang hiin Basunes Lyd . Paa de smfindtligste Steder maatte Hedenstabet cmgribes ; dets indvortets Fuulhed maatte aabcnbares , som Apostlen uden Sky paa hiint og paa saa mange andre Steder ( f . Er . Nom . 1 ) gjorde det . Saa smerteligt det var for Hedningene at hore , naar de klagede over , at den offentlige Ncering og Livets Lyst og Frodighed leed under de Christnes Fcerd , saa maaltc det dog aabenlyst vises , hvad Grnnd hiin Anklage havde , som Tertullian blandt andre gjor i selgende Ord :  leg vil sige det ligefrem , hvilke der i al Fald kunne beklage sig over de Christnes Ufrugtbarhed . Det er fsrst Kodlerne og de , som holde offentlige <b>Fruentimmer</b> , Vadeopvarterne i Vordellerne ; derncest de leiede Snigmordere , Giftblanderne, de , der omgaaes med magiske Kunster ; endelig Sandsigerne , Tegntyderne og Stjernetyderne . Sandelig , at vi ere ufrugtbare for disse , det er vel en stor Frugt " Men ogsaa fra den anden Side , naar Spsrgsmaalet blev om Dyds-Begrebet, som hedenske <b>Vise</b> opstillede , om at prove det ester Aandens Maalestok ( folgelig ester D risternes Rod og Maajets Klarhed og Sikkerhet » ) maatte den uundgaaelige Kamp blive ligesaa svcer . Thi de Christne maatte fordre og fordrede ew sand Dyd , opvceldende as Hjertets Rod , der ydmygede , ei opblceste Mennestet , der ei var en Frugt as egen Indbildning , men Naadcns Gierning , som fornyer os , der ei blot frydede ved sit Skin , men gavnede ved sin Frugt , der ei lod det scedelige Menneske blive klcebende med sine Tanker ved Jorden , men oplsftede ham til Himmelen , til al Fuldkommenheds <b>Maal</b> og evige Kilde . De maatte fordre en Kjerlighed , ligesaa dyb , som inderlig og selvforncegtende i sit Vcesen , udstrsmmet as det ^ Voeld , som Apostlen angiver med de Ord :  Lader os elske Gud ; thi Han elskede os forst " ( 1 Joh . 4 , 19 ) ; de maatte skarpt stille mellem den boleriste Kjerlighed , der kun mener , kun sigter ' til sig selv , og den , som fuldendes ved Ovoffrelse , ved Taalmod , ved et himmelst Sind , den , ifslge hvilken den Christne kan og stal elske sine

<h4>2612</h4>ved Tanken derom ? Og vil man ei erkjende Guddomskraftcn, Navnet i det mindste maa man dog cere , hylde , det Navn , der har fort os frem til saa mange Seire , og sial fore os frem til flere . " Symmachu-s gaacr derpaa den Indvending imode , at jo Constantius allerede havde givet Eremplct i denne Henseende , i det han lyd bortfjcrne Victorias Alter . , / Lad os dog hellere " , siger han ,  efterligne af Constantius , hvad der er Efterligning yard . Fremfor Alt lad os aldrig , glemme , at dette Erempel snart udstettcdes , at Fyrster visselig ei bor tåge ester det , man snart skal angre at have indfsrt . " Han minder om , med. hvor megm Skaansomhed og Deeltagelse Constantius selv , ved sit Ovsvld i Rom , behandlede den gamle Gudsdyrkelse og dens Tilhcrngere , hvorlunde han slet , ikke bersvede Vestalindcrne noget af deres Privilegier , ikke voegredc sig ved at give af Statens Midler til Befordringen af den fcrdrene Cultus ; hvorlunde han med naadigt Blik modtog Senatet , med umiskjendelig Interesse sig om Gudernes Navne , om Tcmplernes Historie , om deres Stiftere .  Og det er Religionsfrihedens sande Orundscetninger " , siger Sy mm ach us ;  < hi som hvert Menneske ved Fodselcn er tildeelt en egen <b>Sjel</b> , saaledes have jo og de forskjellige Folk deres egne Skytsguder ; begge maae vi tlllcegge Evigyedens Attribut . Ia — hvad mere er — hvoraf stal man vel loere i saadanne Ting , hvor Fornuftens Lps er saa dunkelt , uden af Erfaringen ? Og strider ei Erfaringen, mange Aarhundreders Troe og Lykke paa vor Side ? Er det ei , som vi Horte Rom , det . gamle , cervcerdige Rom , staae op og tiltale Folkets Fyrster og Fcedre :  , , 3 Erer dog mine graae Håar ! Jeg vil nu engang leve efter Fcedrenes Viis , og jeg har Ret til det ; thi jeg er frie . , Denne Cultus , som nu er Talen om , har bragt Verden , under mit Herredommc ; med den stog jeg Hamubal tilbage fra Roms Mure , Scnnonerne fra Cavitoliet. Skulde virkelig den bedagede <b>Moder</b> have levet saa lcenge , for at friste sine Sonners Spot ? « "  End mere " (vedbliverSymmachus)  det kommer jo her ikke an paa Veiene ; alle maae vi dog , om og ad forskjellige Veie , komme til eet <b>Maal</b> . Det , Menneskene ere og maae vcere enige i , det er jo kun , hvad de virkelig have tilfselleds , Hunlen , de skue op til .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111103032'target=_>Krummacher, F.W.., 1856, Thisbiteren Elias</a><h4>240</h4>undladt at sende mangeslags Stikbreve eftcr Elias , fMeSceptcrct.  Og cn Enke stal forsørge digder . " En nnderlig TrM . En <b>Kvinde</b> , som selv har mistet sin Forsørger , en phMicist Enke , altsaa m hedensk <b>Kvindc</b> , mod hvis Afgudcr Elias saa skarpt ivredc , og hvilken Enke ? Hvor stal han blandt Landets tusinde Enker finde dm rette ? Nu ja , det var Men en Marschordre as et eget Slags . Heller ikke der var nogen Haand at se . Men kun stille . De fleste Veie , som Herren gaar mcd sine Wrn , ere som Dagen , der fMs as den mMke Nat , i dyb Skumring tåger den sin Begyndclse , men straalcr snart i Morgenrøden, og strider frem fra Klarhed til Klarhed , indtil den fuldc Middag .  Staa op , gaa til Zarcpta , som er ved Sidon " og ved Tyrns , tilseie vi , midt imellem disse to Handclsstceder . Dersom vi overscette denne Stads Navn i vort Sprog , saa fremkommer en smnk og tr ^ stelig Tanke . Harcpta betyder en Smeltehytte . Men i <b>aandelig</b> Forstand er en S m e l t e h y tt e der , hvor Menneskets Kj ^ d bliver undertrykt og korsfcestet , og det forfengelige Hjerte ydmygct og kastet i SiMt . En Slneltehytte er der , hvor Mennesket ser dybere ind i sin Elcndighcd , og hans Afmagt trceder i fornyet Klarhed frem for hans Ainc . En Smeltehytte er der , hvor Trceugfclcns hede Vande stramme over vort Hoved , og Anfcrgtelsens Luer Me os i Troen alene ; hvor den Sclvtillid , som endnu sindes hos os , bliver tilstammc, og enhver Stav , fom ikke er Kristus , gaar istykker lucllem vore Hcendcr , — der , der er Zarcpta , der staar Smeltehytten.  Se " , siger Herren ,  jeg vil smelte og prM dem ; thi hvad sknlde ellers jeg gjMe , da mit Folk smykker sig saa . " Men uaar Gnldct er i Digelen , saa er Smelteren ncrr . Tcrt ved Zareftta ligger Hr ns ; det bctydcr en Klippe , og hvem sknlde ikke vide , hvilkm Klippe vi mene ? Der gives knn en , paa hvilken man kan forladc sig , og som vil beståa , naar alt Andet smelter og opbrcendes , og denne Klippe er lesns Christus . Oui ham siger Malachias :  Han stal sidde ( nemlig ved Smelteovnen ) , at smelte og rense SKvet . " Han skaffer Trcengselen Frngt og en heldig Udgang , og leder Prøvelsen til det rette <b>Maal</b> ; han blceser i rette Tid til Ilden , og dcempcr den igjen , naar dm flammer for stcerkt , han ftrger for , at den knn fortcerer Slagget , men ikke rMr Guldet , og bevarer dem , som ere i Ovnen , at de ikke forsage . O ja , det er meget trMeligt at Zarepta ligger saa ncer ved Tsor eller Tyrus , det er , dcu store

<h4>259</h4>Skyld . Israel har tabt ham , cn arm Betlcrske har vundet ham . Men saa gaar det omsider et Folk , som ikke vil omvende sig til Herren , og reise sig op , hvormeget man end prcediker for det . Brødet bliver taget fra det , og givet Hundene, som smcegte udeufor . Herom advarer ogsaa den Herre Jesus , idet han siger :  Mcn i Sandhed siger jeg Eder : Der var mange Enker i Israel i Elice Dagc , der Himmelen var lukket i tre Aar og sex Maanedcr , men til Ingen as dem blev Elias sendt , uden til Sidons Zarepta , til en Enkekvinde " . * ) Der sidder han nu fornKet med det hele Hus . Al N ^ d er forsvunden . Melct i Krukken bliver ikke fortceret , og Oliekruset fyldes altid paany as sig selv ifølge Herrens Forjcrttelfe . Det aandelige Taffel bliver ogsaa altid igjen dcekket paany . O , hvor den arme <b>Kvinde</b> glceder fig over , at hun saalcdes hver Dag kan sidde ved denne <b>vise</b> , fromme Mands Fadder , for af ham at modtage Undervisning om de himmelske Ting , og hvor gjerne oplader Elias sin Mund til denne elskelige , hjertelige og enfoldige SMerfjcel . De bede sammen , lcese sammen i Mofes og Profetcrne, underholde sig med hinandcn om den forjettede Messias og hans Komme , istemme nu og da en <b>aandelig</b> Sang til Herrens Mre , og Tiden bliver dem aldrig lang , og Guds Engle kunde glcede sig over cn saadan lille Kirke i ørkenen , og med Lyst stue ned til den . Ser , ser , kjckre Brødre , det er Udgangen og Enden paa cn Vei , som bcgyndte i saa dybt MMe . Saaled^s ftrer Gud sine BMn , i det Legemlige som i det <b>Aandclige</b> altid til det herligste <b>Maal</b> .

<h4>509</h4>Og af dem er der en stor Mcengdc , Gamle og Unge , Mcend og <b>Kvinder</b> , KjFbmcend og Arbeidere , Herrer og Tjenere , Mestere og Svcnde . Vredens Kar , opbcvarede til Guds Retfcerbigheds Aabenbarelse paa den store Dag . lakobs Klippe vil falde paa dem og sønderknuse dem . Men vil det da gaa dem bcdre , som forblive leirede mellem Grcendserne , og ville med deres Hjertes ene Halvdel tilhM Baal , og med den anden Jehova ? O , at denne Slcegt af . Haltende blot ikke udgjoide Flertallet af vore Folk ! Men dcsvcerre er det saa . Halten snart paa denne , snart paa hin Maade , hMer til Dagens Orden , og Helhed og Bcstemhcd i det guddommelige Liv er cv sjelden Perle . O ve dig , du hyklerske Slcegt , du som mener at kunne dele din Kjcerlighed og dine Tjenester mellcm Gud og Verden , og snart vakler til denne , snart til hin Side , i den me Time bpier dine Knee for Herren , for i den følgende at bM med Verden . Hvem er det hMste Gode ? Er det Herren ? Hvorfor er han ikke dit Et og Alt ? Hvad stal da denne forbandcde Jagen ester forgjengelige Fordele og Klenodier , denne nfgudiske T ^ rst efter forfengelig Mrc og jordisk Herlighed, denne Engstelige Sorg for Rigdom og Bckvemmelighcd denne Tragten efter verdslige Vellevnet og Fornyelse , og denne hedenske Sorg over timelige Tab betyde ? Lad da Herren vcere din Skat , og han din Sorg , og han din eneste Kjcerlighed og din eneste Lcengsel . Men er Verden dit hFicste Gode , er < det den , som kan gMv dig lykkelig og salig , som kan forlese og triste dig ^ nu vel , saa lev for Verden , og spild ikke din Tid med Sang og BM , med Kirkegang , Bibellcesning og andre gudelige Gjcrniugcr . Vliv dog <b>vis</b> i din Sag , og bliv noget Helt i dit Liv og i din D ^ d . Be Eder , I som halte mcllem Tiden og Evigheden ! I ville tilsidst halte Eder nU i Hclvcdc , det kan aldrig feile . Bestemmer Eder dog . Er Menneskets Liv slnttet i Tiden , og ikke Noget mere at vente paa hin Side Graven , nu vel , saa vcere da Eders LMn : Lader os cede og drikke og vcere lystige ; thi imorgen ere vi d ^ dc " ; nyder da Eders Dage , lader da Eders Begjcerligheder , faa frit L ^ b , og vcerer ingen Narre , at I ville spilde Tiden med de saakaldte Forberedelser til Evigheden . Men er Eders blivende Sted ikke her , gives der en tilkommeude Stad , gives der en Evighed udcn <b>Maal</b> og Ende , hvad skal da Opholdet underveis bctyde , hvad stal det betydc , at man bygger Hytter midt Pillegrimsveien?

<h4>716</h4>Elias stal dF . Det har lesabel svoret ved sine Guder . Da trceder Herren imcllcm , han , som kan lceggc et Aag paa EnhjMningen og fceste en Krog i Leviathans Ncese . Hvad ster ? En paafaldende Omstcendighed . lesabel , som ellers ftger sin Lige i intrigant Troedskhed , kommer , udm selv at vide hvorlcdes, paa den underlige Tanke , fMnd hun udfMr sit Mordanstag, at sende et Bud til den forhadte Profet , og uuderrette ham om hendcs morderste Forscet . Det var uklogt . Vistnok , men hvem kan verre klog , naar Herren gjM ham daarlig og taabelig ! Og han kan — hans Navn vcere priset ! — forvirre og slåa med Dumhed , hvem han vil . Budet sinder Profeten og siger :  Min Fyrstinde lader dig sige : Gudcrne gjMe mig saa og lcegge saa dertil , om jeg ikke scetter imorgen paa denne Tid din <b>Sjel</b> , ligesom en as disses Sjcel ! " — Elias hMr det . Hvad stal det betyde ? Skal han tro sine egne Oren ? — Min Gud ! hvilken uventet Tidende ! — Hvorledes , ^ det er altsaa det endelige Resultat as alt haus Arbeide og Kamp ? Saadant det sidste Indtryk as den halvfjerdeaarige Landeplage ? Dette Frugtcn af de uhørte Nndere og Vidnesbyrd paa Karmel ? — Om nogensinde , saa havde dennegang de gladeste Forhaabningcr draget ham til Isreel . Og nu et saadant Budstab ! — <b>Kvinden</b> mere forstokket end nogcnsinde fFr , Akab igjen af fammc Mening som hin Dgle , Israels Reformation , som Elias allerede havde troet saa ncer <b>Maalet</b> , nu pludselig truet af Modvirkninger, som maaste for stcdse ville hindre den , og han selv omringet af Drre Farer , end nogensinde ! O ! du galdebittre Drik for Gudsmandens Sjcel ! Hvo traster ham over denne uventede , begrcedelsesvcrrdige Tingenes Vending ? Hans skjMne,ste Forventninger ligge ved hans Fadder i Stumper og Ruiner . O ! ve over dette Fdcloeggeude Hagelslag ned i Scedemarkcn , om hvilken han allerede drMte , at den hvidncde til Hesten ! — Nei , nei ! Noget mere Wleligt end dette havde vel ikke kunnet trceffe vor Profet , og dersom hans Tro kommer gjcnnem disse Klipper uden Skibbrud , da ligger det jo soleklart for Dagcu ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013091707012'target=_>Nicolaysen, N., 1858, Norske Stiftelser</a><h4>1390</h4>i en vedholdende Sygdom ej kan andet flutte , end at min Forlosningsstund er ncer forhaanden , saa haver jeg i Jesu Navn besluttet , medens jcg endnu er ved fuld Sands og Siudskrcefter , at fuldbyrde , hvad jeg lcrugc har haft i Tauke , som er , at skjenke en <b>vis</b> Capital til Eker Prestegjelds Skolevæsen , hvortil jeg saa meget heller har ladet mig bevcege , som i ingen Menighed gives saa mange fattige og forsomte Born , baade indfodte og fremmede , som Alle ville bruge til Sloeb og Arbeide , men Ingen holde i Skole , for at forfremmes i Guds <b>Kundskab</b> ; naar nu saadanne ere blevue 16 a 17 Aar gamle , kan de umulig bringes til Confirmcttion, med mindre man vil tage dem i sit Hus , som ikke er Enhvers Lejlighed , eller og maa de scrttes hen i en af de faste Skoler eller til Stedets Klokker , for der at nyde Kost og Undervisning i det mindste en Sommer igjennem , men dertil udkrcrves en Fond , og dertil er forncrvnte Gave af mig bestemt , dog skal her vel agtes : 1 ) at da min Formue er meget middelmaadig , kan jeg ej dertil give mere end 4 UO Ndlr . ; 2 ) at samme Penge aldeles ikke bor henregnes tll Bygdens Skolecasse , siden man af samme ifolge Forordninger ej har ringeste Magt at tage Noget til fattige Borns Undervisning , som alt stal tages af Fattigcassen , uagtet den her og flere Steder neppe er tilstrækkelig til de Fattiges Fode , end sige Undervisning . 3 ) Denne Capital stal altsaa for sig selv henscettes i tareret Jordegods imod aarlig Nente 16 Ndlr . , som efter Stiftets Ordre besorgcs af Stedets Sogneprest , paa hvis Omhyggelighed det og vil ankomme og bero at Hensigten af denne Stiftelse kan opnaaes ved tilborlig Flid i Låsning og <b>Lcerdom</b> af dem , som dette paatage sig , ligesom han og derom indgiver aarlig Underretning og Regnskab til Stiftet . Nogen Nceompcnse bor en saadan Skolelcerer have for sin Umage , og den mener jeg man uden Betænkning kan tage af Bygdens Skolecasse , da samme just er til Skoleholdereus Louniug , og den Udgift kan Eker Skolecasse meget vel taale , da den har en god Fond , som aarlig stiger , saalamgc crtraordinaire Udgifter ved Skolehusenes Reparation ikke indtrceffe . 4 ) Paa denne Maade skjenker jeg af min Boets Lod disse 40 tt Rdlr . , siger sire Hundrede Rigsdaler , til dette gudelige Brug . 5 ) Saa straren jeg er dod , og mit Boets Losore og Ejendom ved Auctiou er solgt og Pengene indkomme , stal denne Capital straren udbetales . Foruden forbemeldte Legat haver jeg sijenket og givet min Vroderson , den unge Peter Strom , alle mine Boger , som ikke ere destincrede til den Deichmanske Bogsamling i Christiania , dog undtages herfra de danske Boger , min <b>Kone</b> kunde behove og finde for godt at udtage . Naar uu efter miu Dod Prestegaardeus Aabod og den paa min og Hustrues fcelles Bo heftende Gjeld og Udgifter ere betalt og for-

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009031312001'target=_>Lous, M.C. Theodor, 1865, De ti Bud</a><h4>221</h4> Kom Sabbatens Dag ihu at helligholde den ; sex Dage sia ! du gjere al din Gjerning ; men den syvende Dag er Sabbat for Herren din Gud , da stal du ingen Gjerning gjyre , hverken du , eller din Ssn , eller din Datter , din Svend eller din <b>Pige</b> , dit Dyr eller den Fremmede , som er i dine Porte ; thi i sex Dage gjorde Herren Himmelen og Jorden , Havet og alt det , som er i dem , og hvilede paa den syvende Dag , derfor velsignede Gud Sabbaten og helligede den " . Den ugentlige Helligdag er altsaa indsat af Gud selv , grundet paa en guddommelig og ikke blot paa nogen menneskelig Anordning : og maa vi i disse Ord , hvormed Gud selv an » ordner denne Ugefest , lcrgge nsie mcerke til dc Ord :  Gud velsignede og helligede den " . At Gud velsignede den syvende Dag vil sige , han gjorde den til en velsignet Dag , en Dag , paa hvilken han iscer vil velsigne og lykkeliggjsrc Menneskene , idet han imr paa den Dag vil aabne os sine himmelske Fvrraadskamre og lade sine uandelige Velgjerninger i rige ? ligt <b>Maal</b> tilftyoe hver den , som selv vil soge og modtage dem , hvilket normere antydes ved det andet Tillcrg , han  helligede den " . Gud helligede den syvende Dag , man kal . der alle de jordiske Ting , hvis Bestemmelse er gudeligt Brug , for hellige Ting : i denne Betydning helligede Gud den syvende Dag ; idet han bestemte den til gudeligt Brug , til Gudstjeneste, til hans Navns Forherligelse , hans Riges Udbredelsc paa Jord . Paa den Dag skulde altsaa Menneskene i Eardeleshed tjene Gud , og derved adstilles denne Dag fra dc svnge Ugeoage , paa hvilke man stal gjore sin Gjerning , altsaa og sa a tjene sit jordiske Behov , Ester dette fla ! altsaa denne Dag vcere en Hviledag for Legemet ; men en Arbeidsdag for <b>Sjelen</b> til dens Helligelse , og saaledes bliver der den lwiestc Forbindelse mellem Hviledag og Hel . I ^ gdag . loderne helligholdt den syvende Dag i Ugen eller vor Ls ' erdag , fordi Gud paa den Dag suldeuoie sit EkabervErk og , som dei heder i l . Moscb . 3 , 3 , velsignede den syvende Dag og helligede den , fordi han hvilede paa den fra al sin Gjeining , som han havde stakt og gjort ; fordi denne Dag senere hen , ved Lovgivningen paa Sinai , med tydelige Ord blev gwet Isderne af Gud , at den skulde verre deres Sabbat , cfr . 2 Moseb . 2 N , 5. . l l ; og endelig fordi da Folket i Mose stdste Dage atter fornyede deres Pagt med Herren ,

<h4>585</h4>i dette , det vigtigfte af Alle , her maa Religion og Bon vare vor daglige Gjast , og fremfor Alt bede vi daglig , at Herren ret vil aabne vor Forstand og vort Hjerte , saa vi ved hans Hjcelp og Naade kunne blive dygtige til at fore et crgtestabeligt Liv , over hvilket hans Velbehag kan hvile . <b>Hustru</b> , Born , Familie , Huus og Hjem , hvormeget skjsnt , hvormeget elsteligt ligger der ikke indbefattet i disse Ord , mine Venner ! de indbefatte i sig Forestillingen om de reneste , de Edleste , de stjsnnestt ' Glceder , og alt dette har Herren vor Gud beredet os i den hellige LEgtestand , som han allerede indstiftede i Paradiset og gjorde til en Velsignelses, en Helligelftlis Ctand for vor hele <b>Uddannelse</b> paa denne Jord ; o hjcelp os da . Herre , at vi ogsaa stedse og altid maa agte denne Stand og bruge den efter din kjcerlige Hensigt , at dm Naade fremdeles kan hvile derover ; lad sand Kjcerlighed boe i vore Huse , lad Nsisomhed , Tarvelighed og Orden smykke vore Boliger ; men fremfor Alt lad Gudsfrygt hellige vore Hjerter , at Fred og Enighed kan boe i Hytte som i Borg , at Bsrnene maa opdrages i Tugt og Wrbarhed, dig til Lov og Priis , dem selv til Fred og Salighed . Ia styrk os . Herre , at vi , selv rensede ved din <b>Aand</b> , kunne fyre Reenhedens , Hellighedens og Kjarlighedens <b>Aand</b> ind i vore Huse og vore Familier , at de kunne blive din Helligaands Templer , der , reiste paa Jorden , vege mod Himlen ng i kjarlig Forening fore Mand , <b>Hustru</b> og Born ind til din Himmelborgs « forkrenkelige Teier , Det skee . Herre , for Jesu Christi Skyld . Amen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014111908036'target=_>Brun, Johan Nordahl, 1841, Johan Nordahl Bruuns Hellige Taler</a><h4>554</h4>at underssge Jesu Dygtighed . Aaron og losva ! I vare herlige ypperste Prcester ; men I begrebe ikke en- hvad Offer denne ffulde frembcere ; I gik ind i det Allerhelligste en Gang om Aaret , og jeg vil troe , med hellig Mrefrygt ; men I forstoge ikke til at aabne den Helligdom , hvor Jesus skulde gaae ind og stndc enevig Forlosning . David og Salomon ! I vare store Konger ; men altfor smaa til at pwve Jesu Færdighed til Strid , og hans dybe Regjeringskunst . David gjorde meget , da han stog Goliat ; men det skulde holde haardere med Jesu Seier , om der var sjunget betimelig: „ <b>Kvindens</b> Sced skal sonderknuse Slangens Hoved . " Salomon byggede sin Guds Navn et Huus , og fik derved et übodeligt Navn ; « len Jesus skulde bygge et herligere Huus , Helvedes Portffulle derin ' od Intet formåa sig , naar Salomons Tempel laae begravet i Asten . Ingen Profets <b>Aand</b> skulde kunne prove Jesu Fcerdighed til at profetere ; thi han var selv den evige <b>Visdom</b> , Guds Mund til Mennesker , han lm ° te os alt Guds Raad til Salighed . Men hvorledes da vide , at han var den Person , som vies skulde ? , « ) „ Ved Vidnesbyrd," og dem havde han baade gamle og nye , baade jordiske og himmelske ; „ Om denne vidne Profeterne , " blandt , dem iswr Esaias , blandt andet iscer i , vor Indgang. Er der forudsagt , hvordan , Jesus ffal vcere , er alt indtil hans Dobeakt , indtil de derved brugende Udtrykke forud beskrevet , og det nu svarede til Beskrivelsen, da er det Vidnesbyrd nok . Men der maatte nu v < ere noget Foregaaende , som skulle vackke opmærksomhed paalesu Person : „ See min Tjener , " staaer der hos Profeten , og der maatte et saadant foregaaende:

<h4>868</h4>Og endelig dette Store ien god Huusholdning: Orden , en sig selv altid lignende Fremgangsmade i alt det Udvortes af Gudsdyrkelsen . <b>Moder</b> ere Pcngeforodere , og idelige Forandringer , endog i ligegyldige smaa Ting , forvirre Almuen , og Guds Ord selv bevares mindre . Ac vcere sig selv altid og i alle Ting lig , klceder en Mand , og i Kirkesager er det et Symbol paa den stadige , sig selv altid lignende <b>Lcerdom</b> . Kun til Exempel : vor Dragt ; ffulde vi ikke gjerne troe , en Anden kunde vccre ffjonnere eller bekvemmere ? men dog elske vi den fremfor enhver Anden , da den nu engang er saadan ; vi skulde elffet den ligesaa hoit , om den havde vieret som Doberens : „ Kjortelen afKameelhaar , og om Lcenderne et Lcederbcelte , " Matth . 3 ; men nu ville vi bytte , hverken med Dsberen eller nogen Anden . Jeg er <b>vis</b> paa begynder man forst at omskabe os , saa vil Forandring folge paa Forandring , vi ville tabe lidt efter lidt Agtelsen for vor Stand , og vi selv , som ogsaa ere sandselige , ffulde fole , at Ktoederne smittede lettelig paa Toenkemaaden ; Sommerfugle i Klceder forst , bleve vi snart i Tcenkemaade saadanne Vendekaaber , om hvilke Paulus : „ De omdrives af hvert Laerdoms Veir , " Ebr « 13 , og da turde Guds Ord tilsidst blive dyrt , da ville neppe Skatten bevares . Men ffulde denne Lcrrers Omsorg blive virksom , maatte den understottes af t » ) Landenes Beherskere . Skoler og Hoiffoler trcenge til Regjermgens Formue , Beffyttelse og Myndighed ; uden disse blev ikke Skatten meddelelig , uden disse blev Læreembedet kun en mislig Proselytjagt ; men at der til Oplysningsvaesenet netop nu af vor milde Regjering arbeides trolig , det falder hver tankende Mand i Vinene ;

<h4>1787</h4>en ny tilsat <b>Lerdom</b> : Lwrdommen om Kristi Himmelfart, om hans Scede ved Guds hoire Haand , om en Talsmand for os hos Faderen : „ Han lever altid , " siger Paulus , „ til at troede frem for os . " Han blev optagen , og nu vide vi , hvor det store Salem er , hvorefter det Mindre paa Jorden skal dannes , hvorhen ogsaa vi skulle komme ; thi han sagde : „ Der hvor Jeg er , der skulle og I vlVre . " O ! I Herrens Forleste ! tcenker paa den Dag , naar Eders Melkisedek skal mode Eder , mere rig paa Velsignelser end den Salems Konge , som modte Abraham . Ogsaa I skulle stride og seire , men blive trcette af Slaget . I blive det ofte her paa Jorden ; Striden er modefuld for mange og Kraften svag ; men det Brod og den Viin , hvormed Jesus <b>moder</b> Eder , er styrkende , at I ikke vansmcegte paa Veien . Jesu sidste Ord og Afskedsvelsignelse gjor dette Maaltid kraftigere end Melkisedeks ; thi det var nok , Ordene han sagde, da han velsignende forlod sine Elleve : „ See , jeg er hos Eder alle Dage indtil Verdens Ende . " Men end Velsignelsen da , hvormed han skal mode os i selve Salem? End naar Abraham , der fik skue den vor him » melfarne Melkisedek ? End naar vi skulle see ham , og alle dem omkring hans Throne , som stred og vandt ? O ! salige Dag ! O ! Mode uden Lige ! O ! Forening over al Beskrivelse ! men imidlertid stutter jeg , som jeg begyndte: giver vor Melkisedek , vor Prcrst og Konge , vor himmelfarne Frelser , Tiende af alt det Gode , han gav os , Tak for hver Gang han bespiste os med Livsens Brod , hver Gang han lod os drikke af Velsignelsens Kalk , Tak for hver <b>aandelig</b> Velsignelse han lod komme over vore Hoveder !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015072308385'target=_>Colban, Marie, 1873, Tre Noveller</a><h4>245</h4>Hovmesterinden var , om ikke <b>Sjelen</b> i Konspirationen, saa dos ; en af dens ivrigste Haandlangere . Hvor sympatbisk end den unge tyske Baron fra første Øieblik bavde været bende , badede bun barn næsten fra det Øieblik , ban optraadte som Beiler ; Iiud kunde blot med Skræk tænke paa , sagde hun til sin Fortrolige, Greven , at den unge giimrende <b>Pige</b> skulde hegraves paa et tysk Slot mellem Ulve ; bun tilføiede ikke — ■ maaske var det alt glemt — at et andet tysk Slot , Grevens eget , og dette ikke ved Neckarens smilende Bredder , men i det vilde Tyrol , bavde fore.svævet hende som alle hendes moderlige Ønskers <b>Maal</b> . Bittert bebreidecle bun sig nu , da den ulykkelige Forlovelse begyndte at gjøre de Kinder blege , bvis Blomster havcle været hendes Stolthed , ikke bedre at have vaaget over sin Yndling , og med hele sin Naturs Ivrighed søgte hun at gjøre sin Uret god igjen . Ingen vidste bedre end hun , at Pauline led , ja at hun ofte græd den halve Nat , og nu vilde hun helbrecie det stakkels syge Hjerte paa sin <b>Vis</b> . det vil sige ved bestandige Adspredelser . Hun bavde seiv kjendt Sorg og var løbet fra den ; nåar hun tænkte sig nøie , nøie om , maatte hun tilstaa , at Sorgen nok egentlig var løben med ; det er en paatrængende Ting

<h4>407</h4>„ De Mænd , der have sat sig i Spidsen for den i mange Henseender berettigede Kvindebevægelse , se ikke , at de forfeile <b>Maalet</b> ved at gaa forvidt . Ved at aabne <b>Kvinderne</b> det offentlige Liv , vilde de skabe nogie Phænomener og en Mængde Latterliglieder . Den overordentlige <b>Begavelse</b> , der skaber Pliænomenerne , er altid Undtagelsen . Den store Mængde vil tro at kunne udrette mere , end den har Evne til , og nåar disse <b>Kvinder</b> nu møde frem i det offentlige Liv , udrustede med stor Indbildskhed og middelmaadigt Talent, vil dette bere de dem ganske andre bitre Liclelser, end den fordums saakaldte Undertrjkkelse . <b>Kvindernes</b> Velgjører er ikke Den , der vil give dem maskuline Rettigheder , men Den , der lærer dem at forståa, hvad der kan gjøres uc ! af Enhvers Hverdagsliv , Den , der lærer dem at elske og løfte det Liv , som nu engang er deres . Nåar navnlig i den store Verden saamange af de Bedste klage over , at deres Dag ikke er udfyldt , er det , fordi de ikke forståa at smykke sin <b>Aand</b> . Istedetfor at fylde sin Dag med Bagateller, der bestandig gjør dem mere tomme , skulde de omgive sig med Bøger , Musik , Pensler , give sig en eller anden stor Interesse . Seiv Videnskaben staar dem aaben , dersom deres <b>Begavelse</b> gaar i den Retning. Det eneste , der ikke passer for dem , er et Liv udenfor Huset , der kræver Hardiesse ; den brændende Sol , der er velgjørende for Solsikken , gjør Rosens Blomstring kort og bringer Violen til at visne . ' 1

<h4>725</h4>af . Doktorens <b>Damer</b> bleve egentlig budne for at de skulde være med paa det aarlige Besøg paa Russeskibet, som dennegang var en større Høitid end ellers , da man var kommen paa en ganske ny Fod med Russen, der , siden han sid st var der , havde havt den store Sorg at miste sin unge <b>Kone</b> , som lod ham igjen med to ørsmaa Pigebørn . Dette var strax en Grund for de trohjertige Folk til at <b>vise</b> Manden mere end den sædvanlige Venlighed . og da han saa havde havt nok en Angst at udstaa for sin gamle Moer , der dennegang var fulgt med paa Reisen , var det bleven Alvor med Yenskabet . Moderen var med et bleven blind , og der , hvor han boede , ved det hvide Hav , fandtes blot en Usseldom af en gammel Doktor , og saa havde Skipperen besluttet sig til at tåge sin Moer med sig til Hammerfest for at opereres . Den gamle Russedoktor kaldte hendes Syge „ den gråa Stær " , og <b>Konen</b> kunde faa sit Syn igjen , sagde han , men seiv turde han ikke paatage sig Operationen . Paa Hammerfest henviste man ham til Doktor Lassen , der trods sin unge Alder havde stort Ry som Læge , især som Øienlæge. Saaledes kom den blinde <b>Kone</b> til Handelsstedet. Hvor travlt de sex <b>Piger</b> end havde , fandt de dog hver Dag en liden Stuud til at løbe ned paa Russeskibet . <b>Konen</b> var en født Finmarking , som Yvoffs Fader , der før ham for paa disse Kysfer , havde hjemført og russificeret saa vel , at hun , nu da hun eftertredive Aars Forløb kom igjen , næsten havde glemt sit Modersmaal. Ofte forstod den Ene ikke , hvad den Anden sagde ; men at pleie Syge gaar i alle <b>Maal</b> , blot Hjerte-

<h4>1381</h4>Præsten greb den Ulle Haand og holdt den i sin , medens han med synlig ftlæde betragtede hende . Alt , hvad Vennen ikke havde fortalt ham , thi Læberne i Norden tale ikke let om Hjertets dybeste Ting , fortalte dette glade Ansigt , som sikrede Vennen bans dybeste Agtelse . En Mand kan lide , en Mand kan have bittre Ting at udkjæmpe med sig seiv . og kan dog føre et stort Liv , være stærk og sund til sin offentlige Virken , mægtigt føle , at Individet og dets Smule Lykke er en ringe Ting ; en <b>Kvinde</b> derimodhar blot sit Hjerteliv , og her havde en barmhjertig Haand aftørret Taarerne . Og aldrig aftørres en Tåare , uden at den , som gjorde det , føler Virkningen paa sig seiv : en Mand , som med Alvor og fast Villie sætter sig det <b>Maal</b> , at hans <b>Kone</b> skal være lykkelig , bliver det ogsaa seiv , om ikke paa hendes Maade ; der drager noget ind i hans <b>Sjel</b> , som Ingen ved at sige hvad er , uden den , som har prøvet det , og som kaldes Fred . Og indre Fred skal der til for den , som skal virke stort udåd .

<h4>1639</h4>forandret sig , jeg ser blot ingen Grund- til at juble over Forandringen . Heclenskabet havde to bestemt udprægede Karakterer : Familiemoderen og Courtisanen . Den Førstes Plads var saa underordnet , hun var saa ydmyget i - Familien , i Samfundet , at Alt , som var skjønt og begavet , valgte Courtisanens Rolle ; men denne gjorde bun idetmindste stor . Hun var Mændenes Lige i <b>Aand</b> , i Kunclskab : Aspasia kunde testamentere Alcibiades et Bibliothek . Uden Indflydelse til det Gode havde hun , ligesom sit Forbillede Venus , de urene Ønskers <b>Moder</b> , Guders og Menneskers Tyran , en forfærdelig Indflydelse til det Onde . Vor Tid har ogsaa to Arter <b>Kvinder</b> : <b>Husmoderen</b> og Modedamen . Af den første er der tusinde Varieteter , alle mere eller mindre ufuldkomne ; den anden driver det til Mesterskab i det miserableste af alle Genre , det saakaldte uskyldige Koketteri . Courtisanen besad med sine Laster dog en <b>vis</b> Storhed ; hun vilde og hun kunde herske og var af Jern . Dette Væsen behager sig i sin Lidenhed . Hun gjør sig en Ære af ingen <b>Kundskaber</b> at besidde og siger uden at rødme , at hun ikke befatter sig med en saa besværlig Ting som at tænke . Hun dyrker blot sin Person og alt , hvad der kan give den Relief . » Alt for sig , Intel for Andre " er hendes Devise . Altfor zart , altfor træt til at stige fire Trapper op for . at besøge en Syg , er hun stærk indtil Forfærdelsé , hvor det gjælder Fornøielsen , passioneret indtil Vanvid for alle Frivoliteter . Uden en Tanke for deres Kval , der have skjænket hende sin <b>Sjel</b> , lader hun sig bedøve af den grove Virak . Verden brænder for Sine ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081303012'target=_>, 1863, Bibelen, eller Den hellige Skrift</a><h4>1049</h4>udeliikke Troende af Hedningcrne , som stutte sig til dem sSml , Matth . 8 , 11. ) . — Synger I Himle ! og fryd dig , du Jord ! Lovsang i is Tilbageblik paa den fuldendtc Forlosning som Cs . 44 , 23. Herren sclv indforcr her Frelsens Sendebud talende M 52 , 7. ) . — Men 14 Zion sagde : Herren haver forladt mig . Ogsaa hellige Sjcele blive ofte anfegtedc af saadanne Tanker i Trengselens Hede . Herren , som har talt fra V . 8 af , indforer her Zion talende og taler derpaa selv videre indtil B . 21. — Da vil jeg , jeg dog ikke forglemme dig . 15 Gnds Kjcrrlighcd ovcrgaar Moderkjcerlighed . — Se jeg tegnede dig 1 6 paa degge mine Hcender ; dine Mure ere idelig for mig . Billedl>gt talt men meget kraftigt . Ligesoni Menneskene , naar de have en Person kjcrr fremfor alt Andet , tegne hans Navn eller Billede paa Haauden , paa Armen eller Panden saalcdes bcerer jeg Zions Navn og Billedet af Gnds Stad og dens Mnre altid i og hos mig . — Dine Bygmestre ile til o . s . v. Ncermest maa dette henfores til 1 7 Jerusalems nye Opbyggelse ; men efter den dybere Mening er denne dog kun Forbillede paa Opbyggelsen af det forklarede Zion , som bliver til evig Tid . — Du > ? al binde dem paa dig som Bruden 1 8 gjsr . Herren lader i Synet Zion skne Israels tilbagevendcnde Forloste. Disse stulle for Zion vcere til en Prydelse , som den er ifort , ligesom en Brud er smykket med pnrgtige Klcrder . — Dine Bsrn , 20 som du var berøvet ; eg : Din Bornloshcds Born . Guds odclagte og forstyrr > ' de Stad ligncde en ufrugtbar <b>Kvinde</b> ; thi i Ddcloeggelsens Tid havde hlm ingen Born og kunde ingen Indvaanere avle . Zions Born . som vende tilbage efter Ddelcrggclsen og befolke Staden , kaldes dens Barnloshcds eller Ufrugtbarhcds Born og Forundringens Sporgsmaal i V . 21 er lost . — De skulle fsre dine Sønner frem 0. s . v. 22 Herren vcekker de hedenske Folk , saa at de selv befordre de Fangnes Til > bagekomst ; saa bliver Zion igjen rig paa Born . — Og bonger 23 skulle verre dine Fosterfcrdre og deres Fyrstinder dine Ammer . Konger og kongelige <b>Kvinder</b> stnlle tåge sig af Zion , fordi dette er til Wre og Velsignelse for dem selv og deres Lande . Fromme Fyrster benytte gjerne deres Magt til at pleie Herrens Menighed saa godt , som de forståa det . Og opslikke dine Fsdders Stsv . De skulle kysse dine Fodder , et Bevis paa Mrefrygt efter osterlandst Skik . Mrefrygten for Herren overfores ogsaa paa hans Tjenere og paa hans Mcnighed , naar Herrens <b>Aand</b> i Sandhed lever og er virksom i den . — Mon det skal tages fra den ' vcrldige , son » han haver taget . 2 l Et nyt Forundringens Sporgsmaal ligesom V . 21. Dette Sporgsmaal stal virkelig blive opkastet ; thi det Furnnderligc stal virkelig ste ved , Herrens Kraft . Over den Stcerke stal en Stterkerc , over Undertrykkeren stal en Frelser , over Kjamipcn ssom ikke i Legemsstorrclse , men i Magt overstrider det menneskelige <b>Maal</b> ) skal dcn almcrgtlge Gnd komme . V . 25. Frelseren minder om dette Udsagn Matth . 12 , 39 ; Luc , 11 , 22 , idet han giver det en dybcre Udlcrgning om Menneskenes Forlosning fra Satans Magt . Paafaldcnde er Udtrykket :  Mon den Netfcerdiges Fanger stulle undkomme ? hvor den Netfcrrdige staar istedet » for den Stcerke sV . 25 ) . Den Nctfa ' rdige kan kun betegne den , som forlcengst har taget Noget i Besiddelse og nu pukker paa sin Be » siddelscsret og forsvarer sin Fordring , som haardnakket gjor sin Erob »

<h4>1294</h4>Dit Lys bli ' r klart din Stjerne opgaar . " Noget af dette har allerede enhver Sjcel , i hvem Zions <b>Aand</b> lever , erfarct i Naadens Tider , det er en Forsmag , hvorved det bliver os muligt at forståa det profetiske Ord . Dit L. ys er kommen og Herrens Herlighed er opgangen over dig . Sml . Es . 9 , 2.  Dit Lys " det Lys , hvis Glands skal oplysc dit Morke , din Guds Lys , Herrens Herlighed opgaar over dig . Det er  Lyset fra det Heie , " hvorom Zacharias forkynder Lue . i , 78 — 78. lesns i Tjenerskikkelsen var allerede dette Lys , men i hans anden Aabenbarelse skal det verre endnn herligere endnn mere utilhyllet. Hans forste stomme i Kjodet var Tegn og Forbillede paa hans andet Komme i Herlighed . Sml . Ap . Gjem . 1 , 11. — Og Hedningerne skulle vandre ved dit Sml . ( ss . 2 , 2 — 3. Hvor der er et Aadsel . der samle Drnene sig ; hvor der er et Lys didhcn sanile Lysets Born sig fjernt og ncrr . — Og dine Dstre opfostres ved Siden . Rigrigerc jaalcdcs som der staar Es . 66 , 12 : dine Dotre  sknlle bwrcs ved Siden . " At berre ved Siden er efter hebraisk Sprogbrug det samme som vorr : at berre paa Arinene . Dine Dotrc sknlle af sine <b>Modre</b> og Ammer boeres til dig paa Armene , ligesom Bornene Mare . 10 , 13. bleve baarne til lesnm . Snll / Es . 48 , 22. Saa stor skal Iveren voere , at Ingen bliver tilbage , men Alle komme til Zion . Disse Zions Sonner og Dotre ere ikke blot legemlige Esterkommere af Abraham , men ogsaa aandelige Abrahamsborn . der ere af hedensk Mt , nien formedelst Troen ere fodte af Aanden . Sml . Cf . 44 , d . Ps . 87 , 4 — 6. Rom . 4 , 9 - 18. - Og dit Hjerte st ' al rceddes og uddrede sig , naar Mangfoldigheden ved Havet omvendes til dig .  Rocddes " egi  bcevc " nemlig af glad Bevcegelse og ndvides saf Gla ' de ; Frygt og Angest derimod gjore Hjertet beklemt). Mangfoldigheten ved Havet ncevncs forst , fordi Landene ved Havet sDerne ) i Vesten betegne de fjerneste Egne og at disses Beboere komme , vcrkker den storste Fornndring og Glcede . Under den videre Udforelse af denne Skildring stilles dog Beboerne af Dsterlandene, der komme paa Kameler , foran som de , der ere namnest ( V . 6 — 7 ) ; derefter folge Vcsterlandcnes fjernere Beboere , der komme paa Havskibe sB . B — 9 ) . — ' veilsbere af Midian og Efa . Vciloberc 9 : Kamelfol , nnge raske Kameler , der lo be hurtigere end de gamle . Alle de af Scheba stulle komme . I Midian 6 fa og Saba ( Seba ) bocde legemlige Ofterkommere af Abraham ved hans anden Hnstrn Ketnra , som han cegtede efter Såras Dod sl Mos . 25 , I — 4 ) . Ogsaa i legemlig Henseende vare disse saaledes allerede Abrahams Born . Som Excmpel , og nden at de andre Broderstammer ndelukkes , blive de derfor her forst nawnte som saadannc . der ved Troen ogsaa stulle blive Medarvinger til Israels Forjettelser . Fra Saba kosti ogsaa engang Dronningen fra Arabicns Nige med Kameler , Guld og Specerier sNrter ) , hvoriblandt ogsaa Rogclse , for at hore Salomos <b>Visdom</b> og fe hans Gjerninger sl ' Kong . 10 , 1 — 10 ) . Men nn sknlle Alle fra Saba komme for at se og hore . Her er niere end Salomo ! Sml . Pf . 72 , 10 - 11. De skulle bcere Guld og Rsgelse osv. Begivenheden i Matth . 2 , 1 — 11 er ikke endnn den endelige Opfyldclse af denne Spaadom , men vel et Tegn paa , at lesns .  den nyfodtc loderncs skonge , " var den , ved hvem tilstdst Guds Hcrlighed skulde

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011062923014'target=_>Kelly, William, 1880, St. Johannes Aabenbaring med Forklaring</a><h4>1564</h4>Port store <b>Maal</b> , nerst efter at nyde Kristus og fryde os i hans KM-lighcd , sknlde va ' re at dyrke , hvad der er ouerensstemmendc med hans Vilje , saa at vi ikte sknlde give et falsk Vidnesbyrd om hvad han er og har gjort for os . Vi cre ikke af Verden ; og saasnart vi falde tilbagc til Naturcns Kildcr , til vor egen personlige Magt , Indflydclse eller Myndighed , have vi forladt den kristne Grund . I Familieforhold at handle efter vor Stilling og Myndighed er en fuldkommen ret Ting . Heller ikke vil Gnds Vclsignelse verre nied dem , som ikke hcrvde den Stilling , hvori Gud har sat her i Verden , som Fader eller som Barn , Mand eller <b>Hustru</b> o . s . v. De kjølige Folelser , hoist vigtige son : de ere , ere dog ikke Alt . Gud maa hoiagtcs ide Anordninger , som han oprcttcr og helligcr . Dette er Ting , som ikke lider noget Skaar ved vor himmelske Stilling ; tvcrtimod , dette giver os en Leilighed til at <b>vise</b> , at vi i Kristo have faaet en ny Kraft ligeovcrfor hvert lovligt Forhold . Men at tåge vor Del som dem , der have Interesse i denne Verden , er en ganske anden Ting og ikke den Stilling , som tilhorer en Kristen ; men snarcre at gaa letteligt over den som dem , der vide , at deres Lod er med Gud i Hunlen . Kristus kommer forat dsmme Verden , som Gnd betragter som skyldig i hans SsnS Blod ,

<h4>2255</h4>Dette viser , at Troen er en meget forskjellig Ting fra den Evne at forklare et Skriftsteds Billede , noget , om hvis almindelige Bctydning og Vished vi endnu klart kunne overtyde os . Dette er i den Grad Tilfcrldet , at om vi endog nu vilde tåge en Kristen og bede ham forklare alle Enkelthederne ved hint Vers — hvad der mentes ved <b>Kvindens</b> Sced og Slangens Scrd , Fiendskabet imellem dem og Sonderknuselsen af Hovedet og Hcrlen , vilde han , om han end maatte verre fuldtommen <b>vis</b> paa , at der talcs om Kristus og kunde forståa den almindelige Mening af det Hele , dog finde en stor Del Vanskelighed i at forklare , hvad hver Ting bctod . Men deri ligger Salighcden ved Guds Ord , at Folk frelses ikke ved at have klare Tanter om det Dunkle , men Gud ved , hvorledes han skal lede enhver Sjcrl , som er frelst , til det rette <b>Maal</b> . Deres Hjerter hvile paa en Kristus , som har lidt for den og fuldstcrndigt har tilintetgjort Tilintetgjoreren . De kunne maaske ikke verre istand til at fremscrtte sine Tanter klart for Andre ; men Lcerlingens Tro kjender maaske Sandheden ligesaa godt forn Lcereren , skjsnt den sidste alene kan uduikle den med overbevisende Klarhed . Dette viser, at endog hvor Gud anvender disse Billeder , er den almindelige Tanke tilstrcrkkelig klar . At forklare dem i det Enkelte ved Ord kunde maaske verre en uovervindelig Vanskelighed for den Sjcrl , som ikke har nogen Tvivl om den almindelige Betydning .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015061048105'target=_>Brontë, Charlotte, 1851, Shirley</a><h4>934</h4>. . De unge <b>Piger</b> , " begyndte MrS . Pryor igjen efter et Viebliks Forlob , idet en <b>vis</b> Undseelse eller Forlegenhed gjorde hendes Tale afbrudt , . isccr de , som Naturen bclvr begunstiget t < rnle sig oft « see hyppig frem imod et Giftermaal som Enden , som <b>Maalet</b> for deres Forhaabninger . '

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006083100008'target=_>Munch, P.A., 1853, Det norske Folks Historie</a><h4>1880</h4>derpaa tilbage til Danmark ; Mdbelred hadde saaledes neppe engang Brug for Olaf og hans Hcer . I England var ftlgelig intet mere for Olaf at gjsre , og han ledsagede rimeligviis , som vi have ncrvnt , sin Svoger Sigtrygg til Tublin , da denne henimod Slutningen af Aaret 994 fordrev Ivar og satte sig i Besiddelse af Rigrt . Men endnu samme Aar blev Siglrygg fordreven paany , og Olaf maa naturligviis have deelt Skjebne mrd ham . Da han altsaa iniet blivende Sted havde , enten i England eller Irland , er det hecl naturligl , at han tcenkte paa at besoge Norge . Hertil kommer , at man ester hans Rrjse fra Irland eller England ikke herer et Ord mere om hans <b>Hustru</b> Gyda ; hun maa altsaa kort for den Tid vcere dod , og ligesom Geira ' s Dod forhen gjorde ham Opholdet i Vendland modbydeligt , saaledes er det ikke usandsynligt , at Gydas Dod , i Forbindelse med anden Modgang , har givet ham Lede til de vestlige Lande . Ut enkelte Omstcrndigheder ved Olafs Tog til Norge , navnlig Thore Klakkas Sendelse og paatcenkte Forraderi , have kunnet ftrvanskes eller overdrives ved at fortcclles fra Mund til Mund , er ikke andet , end hvad vi allerede saa ofte have seet Gxempler paa ved de paa denne Maade overlevende Sagafortcrllinger . Hvad nu serskilt Veretningerne om Olaf Tryggvesson, tildeels . og om Haakon Jarl , angaar , da meddeler Snorre Sturlasson selv i Indledningen til sit Verk nogle Vink om . hvorledes de ere blevne bevarrde og forplantede . Are frode , siger han , skrev , som han selv bevidner, norske Kongrrs Levnetshistorier efter hvad Sidu-Halls Sonneftn Odd Kollsson berettede , og denne havde igjen faaet sin <b>Kundskab</b> fra en <b>vis</b> Thoraeir Afraadskoll , en viis Mand , der var saa gammel , at han boede paa Nidarnes , hvor Staden Nidaros siden blev anlagt , allmde da Haakon Jarl blev drccbt i ) . Henne Thorgeir Afraadskolls Fortcellingrr er dtt vel altsaa, der nccrmest ligge til Grund for hvad Kongesagaerne berette om Haakon Jarls Nod , om Olafs Ankomst til Norge og ovrige Bedrifter . Men vi vide , hvorledes Vegwenheder ofte endog for Samtidige fremstille sig i et urigtigt Lys , og det ' er ikke engang sandsynligt , at Thorgeir Afraadskoll skulde have vidst fuldkommen Vesked om den ncrrmeste Anledning til at Olaf forlod Irland og drog til Norge . Thorgeir var Ojevidne til hvad der stede i Thwndelagen ; han kjendte maaske Thore Klakka , og vidste Besked om hans Sendelse . Men hvad der ellers var foregaact i England , Irland , Orkns , paa Moster , ved Agdenes , kunde han alene have erfaret af Andres Meddelelser . Alt dette fortalte han nu igjen , efter sin Opfatning . og som e " n crldgammrl Mand , til Odd Kollsson ; denne igjen til Are ; Ares Fremstilling

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015072408025'target=_>Berg, Ole, 1871, Prædikener over Evangelietexterne paa Kirkeaarets Søn- og Helligdage</a><h4>164</h4>Alle de , der ncermere kjendte Sogneprcest Berg , ville have faaet Indtrykket af , at han var i Besiddelse af uscedvanlige Aandsevnsr og gode <b>Kundskaber</b> , at han navnlig var en sjelden starv Tomter , der med Klarhed , Liv og Lethed , i Forening med en <b>vis</b> eiendommelig Humor , forstod at give sine Tanker Udtryk . At dette hans skarve kritiske Blik i Forening med hans humoristiske <b>Begavelse</b> ofte gjorde ham noget haard og hvas i sine Domme , staaer ikke til at ncegte , men de , som i hans sidste Aar kjendte ham ncermere , ville ogsaa vide , at han paa Bunden bar et Gemyt , der var bsiet i Aandens Skole og gjennemtrcengt af christelig Forsonlighed . Han var en tro og kraftig Arbeider i sin Herres Tjeneste ; sine Kaldsvligter røgtede han med Nidkjcerhed og stor Uegennyttighed; hans Virksomhed i sin Menighed bar ogsaa mensynlig rige Frugter ; i Erkjenoelse heraf har Menigheden besluttet at reise et Minde over ' hans Grav og at anskaffe et Portrcet af ham til at hcenge i Kirken . Han var derncest en kjcerlig og sm Mgtefcelle , der „ vel forestod sit eget Huus ; " den Mao.de , hvorpaa han omgikkes sin blide og fromme <b>Hustru</b> , ved hvem han hang med den inderligste Kjcerlighed, mindede om Pauli Ord : „ forekommer hverandre med Wrbodighed!" For sine Smaae , hvoraf de 6 — 4 Dpttre og 2 SMner — endnu ere ilive , — 2 vare gangne forud , — var han en kjcerlig Fader og en christelig Opdrager . Han var ogsaa en trofast Ven og er som saadcm favnet og erindret af Mange .

<h4>2155</h4>rKe sin Sorg for sig selv ; thi den Sorg , jeg mener , er af den Beskaffenhed , at man helst vil glemme den , for at ikke Livet ved den stal aldeles forbitres . I ane maastee , mine kjcere Tilhørere ! hvad det er for en Sorg , jeg sigter til . Det er nemlig netop den , som Dagens Evangelium Mder os til at tcenke paa , den Sorg , som den grcedende <b>Moder</b> og SK-gestaren og Ligbaaren stiller os saa levende for Me . For den Sorg er selv den Rigeste ikke fri ; den nager Kongernes Hjerter paa deres Throne , og ingen menneskelig Magt kan bortrydde den eller endog knn mildne den . Den , hvis <b>Maal</b> er at samle Gods og Guld , har foruden alle de andre Sorger og Bekymringer, som den Afgud , han tjener , volder ham , altid ogsaa denne : „ hvorlcenge vil dette vare ? og naar min Tid er omme , hvad saa ? " — og en sand Melse , som han ikke aldeles kan forncegte , siger ham , at hans Skatte ikke kuune baade ham , naar hans Legeme smuldrer i Graven , og en ikke mindre sand og paalidelig Stemme i hans Indre maner om , at det , selv om hans Tanker kunde forsone sig med den for ethvert Mende Menneskehjerte saa forfærdelige Forestilling om fuldkommen Tilintetgørelse og sporløs Forsvinden ved ikke engang er ham muligt for Alvor at fastholde denne Forestilling , hvor sMgelig og utilfredsstillende den end i ethvert Tilfcelde vilde vcere , men at han nFdes til at tcenke sig som tilvcerende efter Do ' den , og hvad kan da det samlede Gods baade den fattige og Sjcel , som ingen andre Skatte har end de , som maatte lades tilbage paa Jorden ? Og den , som lever sor at nyde Livet , maa dog sikkert nu og da foruroliges af den Tanke : „ der kommer en Tid , da du ikke kan nyde mere det , . som nu er din Meste Glcede ; " ja , i Grunden kaster vel en mere eller mindre klar <b>Bevidsthed</b> om denne sørgelige Sandhed stadigen en Slagskygge om alle hans Livsnydelser . Og den , som uden at omtumles af vilde Lidenskaber og uden at foruroliges og cmgstes af Sorger for dette Livs Ncering , lever et roligt oq behageligt Liv , nydende det huuslige Livs stille Olceder ien elsket Families ogsaa en Saadan maa jo stundom gribes af den cengstende Tanke : „ hvorledes er min Lykkes Borg grundet ? kan den trodse Verdenslivets Storme og de verlende Skjcebners ofte saa grusomme Spil ? " I Dagens Evangelium fortælles der jo om en <b>Kvinde</b> , der vel engang havde vceret lykkelig som <b>Hustru</b> og <b>Moder</b> , men nn var hun Enke og hendes SM , den eneste , laa Paa Ligbaaren ; og slige Tilskikkelser ere jo ingenlunde sjeldne i Livet . „ Hvad , om ' noget Saadant hcendte mig ? hvad , om trcengte ind ogsaa i min Kreds og bortrev mine Kjcere , En af dem eller Flere af dem , eller , o Gud ! dem

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013080608090'target=_>Luther, Martin, 1830, christelig Postille</a><h4>3620</h4>stal ophoies og prises iblandt dem ; det er det Offer , der ei udspringer af Hovmod og Driften paa egne Gjerninger , men af et frivilligt og glad Hjerte . Thi saadan Tjeneste og Offer vil Han have ; og dette stal Evangelii Erkjendelse virke i os , at vi ikke blive fnndne som de , der utccknemmeligen forglemme Guds store Velgjerning . Selv Hedningerne, som dog ikke have hcwt hiin Erkjendelse og Guds Naade , have hoiligen fordomt den Last , som man kalder Utaknemmelighed , som var der ingen mere forhadt og stjcendig Synd paa Jorden , og de have sagt , at den var <b>Moder</b> til alle Laster . Dette lcerer allerede Naturen og Fornuften om den Utaknemmelighed , som Menneskene <b>vise</b> mod hinanden indbyrdes : hvor meget storre , stjcrndigere og fordommeligere maa da denne Last vare , naar den oves mod Gud Selv , mod Ham , som , da vi endnu vare Hans Fiender og styldige til Helvedes Ild , har af grundlos uudsigelig Naade stjcenket os , ikke ti eller hundrede tusind Gylden, men Forlosning fra Hans Vrede og den evige Dod , og dertilmed Trost , Fred , Glcede og Liv i fuldeste <b>Maal</b> ; hvilke Goder ikke kunne udtales eller fattes i dette Liv , men staffe os Nok at betragte og begrunde i hiint Liv evindelige « , hvor langt strcrkkeligere er det , siger jeg , naar man bliver utaknemmelig for denne Velgjerning , lig hiin Tjener , om hvem Evangeliet fortcrller , der annammede Eftergivelse for titusinde Talenter , og ikke vilde foruude sin Medbroder hundrede Penninge ! Hvo stulde vel her troe , at der vare saadanne onde Mennester paa Jorden , som bevise Gud saa stor Utaknemmelighed for den allerhoieste evige Velgjerning ? Men hvor og hvem de ere , ja , hvor fuld Verden er af Saadanne , det see vi desvcerre for vore

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016031548116'target=_>Landstad, M.B., 1871, Kirkesalmebog</a><h4>2816</h4>Ord , og giv dem din <b>Aand</b> og Naade , at saadan Sced tan boere god Frngt , paa det de tnnne leve dig til Lov , Wrc og Pris , og andre Folt til Nytte og Forbedring , og Ingen til Forargelse , ved din elskelige Son , lesnm Kriswm vor Herre , som med dig lever og regjerer i Helligaands Enighed , een sand Gud fra Evigbcd og til Evigbed , Amen ! Epistelen , Nom . 12 K . 1 V . Derfor formaner jeg Eder , Vrodre ! ved Guds Baruilmrtighed, at I fremstille Eders Legemer som et levende , delligt og Gud velbehageligt Offer , ( hvilket er ) Eders fornuftige Gudødyrkelse ; og flikker Eder ikke lige med deuue Verdcu , men bliver forvandlede ved Eders Sinds Fornvclse , saa at I maa stjonne , hvad der er den Gnds gode og vcldebage » lige og fuldkomne Vilje . Thi formedelst den Naade , som mig er give » , siger jeg til Euhver iblandt Eder , at han stal itte tamte hoicrc om sig selv , eud han bor at tamte , meu at hau stal tamte saa , at hau er besteden , alt som Gud haver tildelt Eilhver Troeus <b>Maal</b> . Thi ligervis som vi dave niauge Lemmer paa eet Legeme , meu alle Lemmer bavc itte deu samme Forretning , ligcsaa cre vi mange eet Legeme i Kristo , men bvcr for sig cre vi binandens Lemmer . Evang cli nm , Lnk . 2 K . 42 V . Og der han ( Jesus ) var tolv Aar gammel , git de op til Jerusalem , efter Hoitidens Scrdvauc . Og der de havde til « endebragt de Dage , og de gik hjem igjcu , blev Barnet Jesus i Jerusalem , og Josef og hans <b>Moder</b> vidste det itte . Men da de mente , at han var i Nciseselstabet , kom de en Dags Ncise > frem , og de ledte efter dam iblandt Slægtninge vg Kyndiugc . Og der de itte sandt dam , git de tilbage til Jerusalem og ledte efter ham . Og det begav sig efter tre Dage , da faudt de ham siddende i Templet midt iblandt Lcercrne , ( hvor ) ban baadc Horte dem og spurgte dem ad . Men Alle , som Horte ham , fornndrcdc sig saare paa hans Forstand og Gjensvar . Og der de saa ham , blcve de forfoerdcdc, og hans <b>Moder</b> sagde til bam : Son ! lwi gjorde dn os Saadant ? Se , din Fader og jeg ledte efter dig med Smerte . Og han sagde til dem : Hvorfor ledte I efter mig ? vidste I ikke , at mig bor at virre s min Faders ( Forretnui . ger ) ? Og de førstode ikke det Ord , fom ban taledc til dem . Og han gik ned med dem og kom til Nazaret , og var dem nnderdanig . Og bans <b>Moder</b> bevarede alle disse Ord i sit Hjerte . Og Jesns forfremmedes i <b>Visdom</b> og Alcer og Naadc hos Gt ' d oH Menncsieu ? .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014110424003'target=_>Huitfeldt-Kaas, H.J., 1876, Christiania Theaterhistorie</a><h4>2185</h4>Iste Act ( Sc . 1 ) . Hr. Jordan i Schlafrock . Han kan ikke faa ud af sit Hoved den Fru Fløitenspring , som kan blev forliebt i paa Bal masqué paa Hoftheatret . Han vil ikke længer være Kræmmer men Baron , kvortil kans gode Yen Baron Puff , der er fortrolig med Kongen og „ Matskalkerne “ , skal kjælpe kam . Han vil da <b>vise</b> sig ved Hoffet , kvorfor kan kar bestilt en prægtig „nymodisk“ Klædning , og venter nu Skrædderen , som ikke kommen . Skjænder over Skræddere i Almindelighed . Venter Mag . Pancrasius , som skal lære kam at skrive zirlig til sin Masque ; venter sin Dansemester for at lære at gjøre Pas i Forgemakkerne og Hofsalen og gjøre rigtig Reverentz ; venter sin Fegtemester for at lære at forsvare sig , om kan bliver touckeret ; venter sin Musicus for at lære at synge sine forliebte Sange . ( Sc . 2. ) Tjeneren melder Skrædder Knapsnit , som kommer med en Skrædderdreng , der bærer den nye Klædning . Jordan proberer den galonnerede Kjole ; Skrædderen forsikrer , at han aldrig kar gjort en saa prægtig Klædning før ; kan syr for fremmede Prinser og Herrer , der kalde ham mon cker ami , kan tage <b>Maal</b> af en Person blot ved at se kam , uden at der skal blive mindste Feil . Efter megen Snak anbefaler kan sig til videre Arbeide og gaar . Skrædder dr en gen begjærer en Drikkeskilling , Jordan giver kam nogle Smaapenge , Drengen siger „ Tak , velbaarne Herre “ , kvorfor kan faar mere , „ Tak , naadig Herre " , ligesaa ; „ Gud velsigne Dem , Hr. Baron " en kalv Daler ; „ Tusind Tak , Deres Excellence " en Daler . Dette finder han er andre Titler end Husbond , Fa ’ er , Principal eller slet og ret Herre . ( Sc . 3. ) Madame Jordan kommer . Hr. Jordan viser hende sin prægtige nye Kjole , fortæller , at kan skal blive Baron og Excellence ved Baron Puffs Hjælp , og fortæller om alt , kvad kan skal lære , og at Skrædderdrengen allerede kaldte kam Excellence . Han veed kun ikke , kvorledes kan skal faa Skik paa sin <b>Kone</b> . Hun formaner kam til at tænke paa sine borgerlige

<h4>3113</h4>Følelserne ved Din Bortgang ; derom vidner denne Jordefærd og de Taarer , som væde Din Urne , ja de vidne om Dit Værd som Menneske . Men heri og i Dit Hjertes Anliggender staar Du ikke for vor Domstol “ etc. Under Afsyngning af Sangens tredie Vers sænkedes Kisten i Graven . „ Efter Jordpaakastelsen strøede de syugende <b>Damer</b> Blomster ned i den , og medens de tvende sidste Vers af Sangen kleve sungne , reistes efter gammel nordisk <b>Vis</b> en Bautasten med Indskrift : En vold de Falsen. Som den bortgangnes ældste nærværende Ungdomsven ofrede Hr. Grosserer John Collett ham tilsidst med de Ord : Norges Ven ! min Ven ! farvel ! en Krans , som mangeaarigt prøvet Venskab havde hundet . Stemningen ved det hele var saare høitidelig , og upaatvivlelig maatte enhver af de tilstedeværende ønske : engang at vorde savnet og hegrædt som En v old de Fals en ! “ — Et Vidneshyrd om den store Sorg og Deltagelse , Efterretningen om Falsens Død vakte , er den Mængde af Gravskrifter og Digte , hvoraf Aviserne vrimle ; nogle af disse ere anonyme , andre forsynede med Underskrifter , saaledes optræde , foruden den nævnte Lieutenant C . H . Sommer , følgende under Navn : K . L. Rahbek , Pastor N . Wulfsberg , Overlærer L. S . Platou , Procurator P . G . Debes , Student P . Schouning . Wulfsbergs Gravskrift 1 ) var fæstet paa Kisten ; i denne omtales hau som „Kongens og Landets tro Mand , lys af Forstand , kraftfuld af Sjæl , Musernes Yndling , høi i Veltalenhed , brændende for det skjønne og store , utrættelig for at naa det <b>Maal</b> , han opstillede sig . “ Derpaa spørges , om det klart føles , „ hvad han var for Selskahet , for Venuekredseu , for dem , Naturen havde hundet til hans Hjerte ; hvad han var for Norges Land ! hvad hans <b>Aand</b> har virket , hans Stemme har sagt , hans heldige Pen , hans Digterfølelse

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009090403027'target=_>Wang, J. Storm, 1860, Skjebnen og mennesket</a><h4>2587</h4>lgen Person mest higede ester . Til endnu mere at foryge sin Glands , havde hun nu Datteren ved sin Side , der , stille og besteden opdragen , ikke kunde revalisere med sin <b>Moder</b> i , hvad denne ansaa for Stort og Elegant mm som endnu mere gav hende Anledning til at <b>vise</b> sine egne Fuldkommenheder og Fortun ved hvert Bieblik ot rette hende og indprente hende Sands for de Fuld- Immenhedcr , som hun ansaa til Pryd for en ung Da- U , der skulde gjere sin Lykke i Verden . Datterens Lkjenhed var hende hsist velkommen , herved vilde hun sm <b>Moder</b> ogsaa komme til at erbolde en vigtig Plads st ! » blandt Ungdommen , thi hun lvivlede ikke om , at en Ltrem af Tilbedere strar vilde omringe Amalie , ligesom « t de unge <b>Piger</b> i hendes afsidesliagende Fedeby vilde lappes om at have hende til Forbillede , naar dun Mst wer den moderlige Haand havde erholdt den stsrste Fllldkommenhed af <b>Uddannelse</b> . Til sine intime Venin-

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006082200020'target=_>Bang, Anthon, 1878, Fortællinger for Folket</a><h4>972</h4>jeg mig mange Gange . Min Stilling ombord forbedredes imidlertid, efter at Proviantforvalteren havde opdaget , at jeg kunde skrive og regne ; han bad mig ofte fri hos Ofiicererne , for at jeg skulde hjcelpe ham med hans Regnstaber , og han fattede tilsidst saadan Godhet» for mig , at han , da jeg , cfter at have udstaaet den gule Febcr i Vestindien , var altfor svag til at gjsre Tjeneste , fik udvirket Frihed for mig til at vende tilbage til England hvorhen han medgav mig Anbefalingsbreve til sine Venner med Opfororing til at hjcelpe mig til en for mig passende Virksomhet » . Vel var mine Tanker sent og tidligt i Hjemmet , men jeg kunde ikke bekvemme mig til atter at <b>vise</b> mig der i den samme hjcelpelpse Forfatning , hvori jeg havde forladt det . Jeg tcentte , naar jeg kom til England idetmindste at kunne strive hjem , at jeg nu var i en ordentlig Stilling ; men Skjcebnen havde bestemt det anderledes ; i Sigte af den engelske Kyst blev vi tagne af en fransk Krydser og bragt md til Havre ; derfra blev Skibets Besetning fyrt som Krigsfanger til det Indre af Landet , og jeg forblev der i 4 Aar , uden at jeg kunde faa give min Familje nogen Underretning om mig . Guvernøren i den Fcestning , hvori vi var indestuttede , var en meget god Mant » ; han lod os nyoe al den Frihed , som lod sig , forene med hans Ansvar for vor Tilstedeværelse , og da jeg temmelig snart lcerte Sproget , som jeg i Skolen havde lcest , talte han undertiden med mig og fattede saadan Tillit » til mig , at han , paa mit Lyfte om ikte at ville flygte , tog mig i Huset til sig og lod mig oplcere i Gartneriet af sin deri meget duelige Gartner . Jeg havde det for en Fange at vcere scerdeles godt , men lcengtes naturligvis efter Frihed , den jeg dog ikke fik fprend , som sagt , efter 4 Aars Forlpb ved Fredsslutningen mellem Frankrig og England . Ankommen til England gjorde jeg straks Forespprgsel om min Velynder , Proviantforoalteren ; han var deo , og da de Anbefalingsbreve , jeg havde faaet , var forkomne ved min Tilfangetagelse , var der intet Andet at gjpre end at tåge Hyre som Matros med et Skib , der gik til mit Hjemsted . Efter ncesten ti Aars Fravcerelse stod jeg atter i mit Hjem , lige fattig som da jeg forlod det undtagen paa Erfaringer . Min Fader var dpd , og min <b>Moder</b> levede i Ensomhet » og Fattigdom . Vort Gjensyn , om det kan kaldes saaledes , thi hun var ncesten blind , var mindre end glcedeligt . Hvad skulde jeg nu gribe til ? Der var nu ikke lcenger Spsrgsmaal blot om mig selv ; det var nu en hellig Pligt for mig at ftrge for min <b>Moder</b> . Hvormangen Gang har jeg pnsket , at jeg aldrig havde lcert Andet end et simpelt Haandvcerk ! Mine <b>Kunskaber</b> var mig blot til Hinder .

<h4>1035</h4> Min gode Larsen , " vedblev Prcesten ,  det stal tidlig kryges , som god Krog stal blive ; det er i Ungdommen Mennesket modtager hine Indtryk og hine lykkelige eller ulykkelige Tilbyieligheder , der fylger ham igjennem hele Livet . Vi byr bestandig gjentage for Barnet : et <b>Maal</b> og en Vilje , og stadigen holde det for Bie , at uden et <b>Maal</b> og en Vilje er Mennesket Intet , udretter Intet og opnaar Intet — Sysselen — Haandteringen eller Kaldet er blot Midler til at na a Enden , , men disse Midler er almcegtige , dersom vi bruger dem med Udholdenhed , dersom vi . forstaar at udbringe det mest mulige ved dem . Indbild Dem ikke , Larsen , at det efter en <b>vis</b> Alder er umuligt at erhverve denne Vilje , hvori vor hele Styrke bestaar . I Ungdommen maa vi , for at danne en Vilje , adlyde ; i den modnere Alder behyver vi , for at give os en Vilje , blot at ville . De , Larsen , f . Eks . har tabt Deres Ungdomstid og luailgm Leilighet » , som tilbyd sig . Lad Dem nu advare af Deres tidligere Vildfarelser . Loer at ville , og De og Deres Familje vil nyde den eneste sande Lykke , som gives herneden . Hav en fast Vilje , og De vil anvende de Kunstaber og Fcerdigheder , De bestoder , til at naa <b>Maalet</b> . Hyr , hvad jeg vil forcflaa Dem : jeg vil overlade Dem et Stykke godt Rydningsland , passende til Deres Familjes Underholdning , naar det engang bliver opdyrket ; De stal ikke savne Midler under en eller anden Form til at dyrke og bebygge det ; — det vil koste Anstrengelse og Myie , og der vil medgaa Aar , fyreno De saavidt har naaet <b>Maalet</b> , at De har et Hjem , hvorhen De kan spre <b>Kone</b> og Bern ; men har De fyrst kjcempet saa langt , saa tror jeg at turde indestaa for , at De har erhvervet Dem en Vilje , der ikke vil svigte Dem for Resten af Livet . De fer nu Maalei , Larsen ! nu udfordres kun Deres egen Vilje . Overvei mit Forslag ,

<h4>1041</h4>ikke ; min Velgjyrer mcerkede det og sagde til mig : Knut , Du har nu faaet en <b>vis</b> Grad af <b>Dannelse</b> og Smag for et mere forfinet Liv ; det vil nu falde Dig tungt at tåge fat paa Plogen . Synes Du ikke om Theologien , saa vcelg et andet Studium ; se Dig vel om , inden Du bestemmer Dig , men har Du engang bestemt Dig , da bor Intet bevcege Dig til at forandre den tagne Beslutning ; din Fader har Intet af efterlade Dig ; din <b>Moder</b> vil efter al Sandsynlighed lcenge overleve din Fader , og Du har endeligen en uforsyrget Syster ; Du har hellige Pligter mod disse at opfylde ; saalcenge Du med Fasthet » holder ved din tagne Beslutning , stal jeg staa Dig bi af yderste Evne ; har Du engang Eksamen , vil Du snart kunne faa Kald og vcere min Hjcelp foruden ; betcenk Dig vel , raadfyr Dig med dine Forceldre og bestem Dig saa " .

<h4>1528</h4>Halvor var neppe fcerdig med sin Fortcelling , da vi naaede hans Hjem , der saa hyggeligt og velpyntet ud . En liden Bygager og en Potetesager omgav Husene , der bestod af , en Stue og et lidet Ws . Et <b>Maal</b> omtrent med vakkert Eng lovede Sauerne god Vintertraktement. Udenfor Huset legede et Par smaa Bprn i den bare Skjorte . Jeg standsede udenfor Grinden og kaldte Bsrnene hen til mig , men de fiygtede forstrcekkede md i Stuen .  Vil Du ikke have en Skaal Melk ? eller kanske Du ikke liker Faaremelk ? " spurgte Halvor , idet han aabnede Grinden . Jeg modtog Tilbudet , da jeg var nysgjerrig efter at se , hvorledes den Mands Hjem saa ud , der kunde paastaa , at han ikke trcengte til Noget . Jeg steg altsaa af Kariolm og fulgte Halvor md i Stuen , hvor jeg fandt hans <b>Kone</b> siddende i Vcevstolen , bag hvilken Bsrnene havde krsbet i Skjul . Der var rent og ordentligt i Stuen , Bord og Bcenke velfkurede , og Sengen med sit Straaunderlag og Skindfceller saa bedre ud , end man er vant til at finde dem paa saadanne Steder . Medens <b>Konen</b> gik ud efter Vand , som jeg foretrak for Faaremelken , satte jeg mig ned , og det lykkedes mig ved et Par blanke Splvpenge at lokke Bprnene frem fra sit Smuthul . De faa funde og friste ud , men Skjorterne var , som man kan tamte sig , ikke meget hvide .  Du siger , min gode Halvor " , begyndte jeg ,  at Du ikke treenger til Noget ; men jeg er <b>vis</b> paa , din <b>Kone</b> er af en anden Mening " . Han lo og svarede Intet , da <b>Konen</b> i det samme kom md.  Du har ret en underlig Mand , Mo ' r " , sagde jeg .  Han vil indbilde mig , at han ingen Ting treenger , og det har jeg aldrig hyrt noget Menneske sige fpr . Du er vist fornuftigere end saa og har nok Et eller Andet at pnste , som jeg gjerne vil skaffe tilveie , hvis det staar i min Magt , da Halvor har gjort mig en Tjeneste , som jeg gjerne vil <b>vise</b> mig erkjendtlig for " . <b>Konen</b> saa sig et Bieblik om og svarede derpaa uden Betcenkning :  Du stal have Tak , som byder , Fa ' r , men jeg ved rigtig ikke , hvad det skulde vcere , jeg skulde ynske mig . Jeg har alt , hvad jeg behyver , takket vcere Manden min " . Var jeg forundret fyr , blev jeg det endnu mere nu ; et <b>Fruentimmer</b> , forn intet Bnske havde , det var da altfor besynderligt . Halvor betragtede mig smilende og tilkastede <b>Konen</b> et venligt Blik .  leg ser nok , jeg faar hjcelpe ler paa Vei , godt Folk , " sagde jeg halv « ergerlig .  Fsrst og fremst vil jeg pnske , al Ungerne der havde Klceder paa Kroppen " . <b>Konen</b> svarede ikke et Ord , mm gik hen til et Stab og tog frem et

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112200010'target=_>Sundt, Eilert, 1857, Om sædeligheds-Tilstanden i Norge</a><h4>585</h4>den udtrykkelig og i Vidners Paahor havde modsat sig . Men man kan voere <b>vis</b> paa , at i selve Livet har den gamle Toenkemaade og Skik trods Loven vedligeholdt sig lcenge , selv til henimod den nuvcerende Tidsalder . Hvad har — ncest Ploining og Ho-Slaat og andre daglige Fornsdenhedcr — opfyldt den norske Almues Liv i disse sidste Aarhundreder ? Mon krigerske Bedrifter, hvormed Wrgjcerrigheden kunde soge sit <b>Maal</b> ? Nei . Mon politiske Kampe , hvor Herskesygen kunde haabe at findc Tilfredsstillelse? Nei . Men de , som havde Magten , de Wldre , Familie- Foedrene og Gaardeierne , besijcrftigede sig med en evindelig Gjentagelse af de mange smaa Optrin , som Trcetter ( Procesfer ) og Giftermaal gave Anledning til , og hvorved de Djerve og Klogtige psnsede paa at faa underlagt sig flere Gaarde og Herligheder. — Denne samme Tcenkemaade . hvorefter de Wldre ssgte at lede Giftermaals-Sagerne til sin Fordel , smittede ogsaa altfor vfte over paa de Unge . Det blev en Talemaade , at det at gifte sig , det var det samme som at forbedre sine Kaar . Det var engang , heder det allerede i den forncevnte Saga , at de to VrodreHoskuld og Rut droge til Thinget , og da talteHsskuld til Rut saa : " Jeg vilde gjerne , Broder , at Du skulde forbedre dine Kaar og soge Dig en <b>Kone</b> . " Rut svarer : " Jeg har lange havt det i Tanke , men vceret uvis ; nu vil jeg dog gjore , som Du synes . Men hvor skulle vi forsoge ? " Og nu berettes der meget omstcendeligt , hvorledes de gik hen til en Mand ved Navn Mord og anholdt om hans Datter Unne . <b>Pigens</b> Fader , som lovede stor Medgift , fordrere stor Algave fra Frierens Side ; men Hoskuld , der som Ruts Talsmand forte Ordet , paatog sig at svare for Alt , og faalcdcs kom en for Rut fordelagtig Handel istand . — Denne Fortolling kom jeg ellers til at tcenke paa under en Samtale med en Bonde , som nu sidste Vaar siydsede

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009080603013'target=_>Sagen, Lyder, 1861, Læsebog i Modersmaalet for Børn og Ungdommen</a><h4>3003</h4>Walter Scott og Cooper have ikke strevet for Born , end ikke for Inglinger , og de , som ttoe , at Bsrn og Inglinger kunne have sand Interesse for disse Forfatteres Vcerker , have selv ingen. Det gaar Born endnu vcerre , end det gik en <b>vis</b> Overscetter af Walter Scott ; det egentlige Charakterististe ved ham , hans Skildringer gaae dem formeget i det Brede ; de forkorte ham ligesom hun Oversetter : de gaae forbi Alt , hvad der ikke behager dem , og Intet behager dem , , uden hvad der bringer dem <b>Maalet</b> ncermere, det er : hvad der giver dem hurtig Oplysning om Helten faar <b>Heltinden</b> ; de lcese for at see Udfaldet , og forat komme efter det , ere de for ivrigt bestjceftigede med Fabelens Traad , til at de skulle kunne standse og selv anstille Betragtninger eller idetmindste lytte til dem , som Forfatteren anstiller for dem . At denne min Anke er grundet , tcenker jeg mange af Walter Scotts Lcesere ville fole sig overbeviste om ved at erfare , at nedcnstaaende herhen

<h4>4771</h4>Tie de sa a lidet med sig selv , felger deraf , at de endnu mindre tie med Andre . De fortcelle , som man siger , Lost og Fast , og alt det uden Hensigt , uden Ondskab ; deres eneste <b>Formaal</b> er at lade Munden lsbe . Man har i gamle Dage havt saa lidet Galanteri , at man har beskyldt det smukke Kjon , iscer ester en <b>vis</b> Alder , for denne Feil . Var det end saa , og gjcelder det maaske cndnu om <b>Fruentimmer</b> af liden Sjcel og slet Opdragelse , da kommer det udenivivl deraf , at disse ere opdragne i en liden Kreds , hvor der er ingen Ting at tie med. Derimod fremstiller Historien os mange Erempler paa , at Hemmeligheder af Vigtighed have vceret tryggelig gjemte i hsimodige <b>Fruentimmers</b> Varm . Imidlertid staae de dog engang i saa ondt et Ord , at Cato den celdre , som rigtignok ikke var galant , men som derimod var saa god en Wgtemand , at han satte storre 3 Ere deri , end i at vcere en stor Senator , ikke desmindre regnede det blandt de tre Ting , han sebreidede sig , at han engang havde betroet sin <b>Kone</b> en Hemmelighed .

<h4>4874</h4>hun ; iscer ssgte hun at vinde hans ganske Fortrolighed , og hvor inderlig denne maa have vceret , kan sluttes deraf , at saa ofte han , endog i sit Livs Aften , saa med et taarefuldt Oie hen til sin Moders Aske og velsignede den , fandt han et Slags Lindring og Trost i den Vevidsthed aldrig at have forebragt hende nogen Usandhed . Saa omhyggelig Moderen var , saa flittig og lcervillig var Ssnnen . Neppe sexlen Aar gammel er han allerede fuldkommen forberedt til Universitetet , hvorom han ved sin Indstrivelse aflagde de herligste Prover . Allevegne hvor Noget kan lceres , er Lmdorph utrettelig . De to fsrsle Aar besogte han daglig de akademiske Forelesninger ; han havde ligesaa meget as Lydighed mod sin <b>Moder</b> , som as egen Tilboielighed , valgt det juridiske Studium ; og han havde det Held , at han tidlig fik Leilighed at indsamle de <b>Kundskaber</b> , hvorefter han higede . Han var atten Aar gammel , da man fandt ham sindig nok og taus nok til at vcere Secretair i det danske Cancelli . Blandt de Professorer , som paa den Tid sirede vort Universitet , var den grundlcerde Gram og den udodelige Holberg ; disses Naad brugte han , og derfor lagde han saadan Plan til sine Studeringen , hvilken han siden saa vel udfsrte . Han indsaa , og hvor sandt er det ikke ? at de Gamle have opfundet og dyrket alle de Vivelisiaber , som hore til at danne Smagen og forskjonne den menneskelige <b>Sjel</b> ; at de , som Digtere , Talere , Philosopher og Historieskrivere vare komne til saadan Hoide , at de maaske undertiden kan efterlignes , men aldrig overtrceffes . Disses Skrifter blive da de Kilder , hvoraf han skal ose , ved disse store Msnsteres flittige Lesning vil han lere at tcenke og skrive . Og sandelig alle Omstcendigheder bidroge til at Planen kunde udfores og at den blev det . Thi alle de Hindringer , som ligge Andre imod i at gaae , om jeg saa maa kalde det , denne rette Kongevei til Loerdoms Tempel , fandt her ikke Sted . Er det Mangel paa Nemme ? Lmdorph havde det herligste Pund . Er det Ulyst hos den Unge ? Lmdorph tsrstede ester <b>Kundskab</b> . Er det Fattigdom , eller Foreldrenes Sparsomhed ? Lmdorph eier det Fornodne og hans <b>Moder</b> giver gjerne . Er det Savn af smagfulde og duelige Veiledere ? her ere Holberger og Rostgaarder og Grammer .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016042848154'target=_>Ahlfeld, Friedrich, 1866, Prædikener over Kirkeaarets Evangelier</a><h4>644</h4>ere dine gamle Skjodesynder dig en Gru , sorger dit Hjerte i dyb Sorg i dagevis , miar du dog har ladet dig besnære af dem , stjælver der euduu efter Synden , ligesoni et Blad , som en raa Vind har berort , bcever lamge efter at Trcekt ' en er forbi : da er han nede ast Bjerget . Driver dn din din Hnsholdning , din Bornetugt i Jesu Ehristi Navn : da er han nede af Bjerget . Bcerer du din Smerte , din Lidelse til hans Wre , glæder du dig i dine gode Dage i Herren : da er han kommet paa Sletteu , ind i Livet . Men han maa komme ued , stal virkelig nyt Liv trives i dig . Dn ved , naar Solen gaaer op ' , forgylder den forst Bjergenes Top . Det seer hel oeiligt no ; men deroppe vurer Intet . Derved voxer intet Korn paa Marken og ingen Frugt paa Trceet . Derved opgaaer forst Haabet hos os om , at den vil komme ned . Forst naar deus Straaler komme ued paa det lave Land , da fremkalder den gode Spirer , da modner den Korn og Frugt . Og just saaleoes er det ogsaa hos os . Naar vi tjene Christus meo Forstauden , men , Livet Intet ved deraf , da er dette ogsaa kun et S tin om Bjergtoppene . Derved trives ingen Retfærdighedens Frugter , ingen Barmhjertighedens Gjerninger . Vi se efter Herren , men vi folge ham ikke efter . Naar du ellers har ertjeudt Noget i din Forstand , f . Ex . at dstte eller hint er hensigtsmæssigt i din Husholdning, i dit Kald , hvor hurtig stiger ikke da ( irt ' jendelsen ned i Hjertet , i Villien . Erkendelsen bliver til Gjerning . Men Erkjendelseu af vor Frelser forbliver ofte i mange Aar , ja igjenuem hele Livet , en blot Erkjeudelse , og bliver itke Mere . — Kjcrre Ehristne , I have Allesammen ofte været i Kirke . I have opbygget Eder ved mangen Prædiken og have maaste siden rost den : ~ Den gik ret til Hjertet , den traf rigtig ! " Men naar I kom hjem , antog da Eders Liv en anden Skikkelse og Farve ? Saa man da , at Christns havde rort Eders Hjerte ? I de fleste Tilfcelde maa I svare : „ Nei , jeg istemmede strax igjen min gamle <b>Vise</b> . Det var siden som for . Hjertet forblev i Verden . Herrens Frygt hang son : en los Lap paa det gamle . ' . vla ' debon . Lognen forblev . Ogsaa mit gamle Va ' sen mod <b>Kone</b> og Born , mod Forcelore , Foresatte og Undergivne forblev som det var . Folk , som jeg forhen hadede , dem hadede jeg fremdeles . Prædikenen havde haget mig saare vel , men Alt blev ved det Gamle . " — Da have I været med Christus paa Bjerget , men han er ikke kommen ned med ' Eder . — Og dog er dette det allerforste Fremskridt , deu allerforste Væxt , som man maa se paa Troeu . Deu Herre Christus er det hele nye Menneske . Han er kommen forat fornye det hele Menneske . Han sad ikke som en lærd Herre paa Kathoderet , forat foredrage en ny <b>Visdom</b> for Folk ; slog saa Bogen igjen og gik atter hjem . Nei , han drog

<h4>1136</h4>Drag til Christus ! De forlode <b>Hustru</b> og Barn , Arbeide og Hus , forat vaudre med. Te glemme derved at forsyne sig med Tcering for Veien . De have vel blot villet gaa een Dag med , men Draget til ham er saa stcerkt , at de ikke kunne komme fra ham . De fom igaaraftes vilde gaa hjem , ere endnu idag hos ham forat stue hans Ansigt , forat hore hans livsalige Tale , forat fe hans Gjerninger . Dyrebare Vrodre og Sostre , hine drog en übekjendt Anelse til ham . De vidste endnu ikke tilfulde , hvad de havde i ham . Og du ved , at du i ham har Forsoning ved hans Blod , nemlig Syndernes Forladelse . Du ved , at du i ham har al Naadens Kilde , Hjertefreden , Husfreden , Trosteren uuder Korset , den aabne Der til det evige Liv . Drager det da ogsaa dig saaledes ? Kan du ikke komme fra ham ? Maa du efter ham , om du saa end derved skulde komme i Hunger og Urken ? En Plante , som du scetter i dit Vindue , den vender snart sine Blade og sine Blomster mod Solsiden . Og vender du den om , forat Blomsten stal vende ind mod din Stue , saa kan den dog ikke lade det voere ; Bladene vende sig atter , og inden saa Tage se atter Blomsterne hen mod den Kant , hvor Solen staaer . Det gjor den , , om den end derved scettes tilbage i sin Vcert . Og du er Guds « Meste , dyrebareste Plante paa Jorden , hans Mres Blomst . Tin Frelser er Solen . Kan da heller ikke dn lade det være , maa da ogsaa du altid vende dit Ansigt mod den ? Og om end Verden med sine Tillokkelser vilde dreie dig om , forat du stuloe fe ind i den , maa du dog alligevel om igjen til Retfærdighedens Sol ? Er det endnu ikke faaleoes med dig , faa gaa ud og sporg Blomsterne vaa Marken og Grcrsset . De ville lære dig det . Sporg disse Skarer af Israel , de ville forkynde dig det . — Hvo der nu folger dette Herrens Drag , han kan derved for en Tid komme i fvar Trcengsel . Hine 5000 vare gaaede efter ham . De vare med ham komne ud i Brkenen . Da griber ørkenens Hersker , HUugeren , dem med sin haarde Haand . Men forend de tcenkte oerpaa , tcenker Jesus derpaa . Forend de begynde at klage , rores hans Forbarmelse mod dem . Forend de soge efter Raad , foger han Raad . Glæd dig , Christen , alle dine Sorger ere bessrgede i Himlen , forend du sorger derfor . Al din Smerte er folt af din Frelfer , forend du foler den . Men han vil ved denne Nod <b>vise</b> sine Disciple , hvor aldeles Intet alt Mennesteraad er , hvorledes Guds Raad alene viser Vei og Seier . Med vor Forstand , med vor Fornuft komme vi aldrig igjennem . Gamle fromme Mænd have sammenlignet den guddommelige <b>Visdom</b> og vor Forstand med Solen , og Maanen . Maanen skinner ogsaa . Den oplyser ogsaa Menneskenes Veie . Man kan prcrgtig vandre under dens Straaler . Men det er blot ved Solen , den er lys . Hvor

<h4>1264</h4>deres Styrke er han alene . Den fromme Spener , der i Begyndelsen af det forrige Aarhunoreoe virkede i Dresden og Berlin i stor Velsignelse for Sjælenes Redning , havde en rigt begavet men vanartet Son . Alle Kærlighedens og Alvorets Midler havde været frugteslose. Endelig var der intet Andet igjen for den bedrovede Fader end at bede for sit Barn . Han raabte til Ond , at han maatte redde hans Son . Veientz dertil overlod han til hans <b>Visdom</b> . Nogen Tid derefter blev den forlorne Son heftig syg . Ugevis laa han næsten stum og übevcegelig 1 store indre Kampe . Paa engang hcevede han med Magt Hcenderne og raabte af det sammenpressede Bryst : „ Min Faders Bonner omringe mig som Bjerge ! " Snart derpaa gav Sygdommen efter . Han fandt legemlig og <b>aandelig</b> No . Snart kunde hans Fader sige : „ Min Son er legemlig og <b>aandelig</b> reddet . " Endnn for sin Dod saa han ham som en vcerdig Mand i et velsignet Embede. — Dyrebare Venner , have vi ogsaa saaledes bedet for vore Born , at disse i haarde Anfægtelsens Stunder maa sige : „ Min Faders , min Moders Bonner omringe mig som Bjerge ! " ? Have vi endnu ikke gjort det , saa lad os gjore det idag . Og naar vi sporge „ For hvem ? " Just for vore Born . Christi Menighed , det er ogsaa dine Born . Og Mange er der iblandt dem , for hvem ingen Fader , ingen <b>Moder</b> mere beder . Nu , disse stulle den troende Menigheds Bonner omringe som Bjerge , som de ikke kunne komme over , naar Verden , Kjod og Djævel ville rive dem ud af den evangeliske Tro og af Guds Borns hellige Tugt . — Nonnerne ere Kirkens Hcrr til dens Redning i det Store og Hele , til Redning af de enkelte Sjælo . Og den der throner blandt Israels Bonner , det er denne Hcers Fyrste , Han vinder Seier .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014060208012'target=_>Luther, Martin, 1878, Husandagtsbog</a><h4>849</h4>Her ser vi den rette Troeskamp . Hvor haardt end Herrens Ord lyder , saa udtyder Maria alligevel ikke alt dette som Vrede , men bliver fast i den Mening , at han er god , og lader ikke denne Mening fratage sig ved dette Stod , at hun skulde holde ham ogsaa i sit Hjerte ikke for god og naadig , som de gjor , der er uden Tro og falder tilbage ved fyrste Stod og ikke tror mere paa Gud , end hvad de foler . Havde Moderen ladet sig afstrcekke ved disfe haarde Ord , faa vilde hun have gaaet stille bort . Men nu befaler hun Tjenerne at gjore , hvad han vilde sige , og beviser , at hun har overvundet Stpdet . Hvad mener Du , naar et Menneske i sin Nod , ifcerdeleshed i Samvittighedens hoje Nod , fik dette Svar af Gud : Hvad har jeg med Dig at gjore ? Forsage og fortvile man han , dersom han ikke tjender scmdanne Gnds Gjerningers Art og er ovet i Troen . Derfor er dette Stykke af Evangeliet det Hojcste , og vi stal vel MErtc os , at vi maa give Gud LEren , at han er god og naadig , om han end anstiller fig anderledes , og om end alle Sandfer og Folelser tcenkte anderledes . Thi dermed bliver Melsen dodct og det gamle Menneske tilintetgjort , for at den rene Tro paa Guds Godhed og ingen Folelse stal blive hos os . Her ser Du , hvorledes Moderen beholder en fri Tro , til et Forbillede for os . Hun er <b>vis</b> paa , at han vil vcere naadig , vist er det ogsaa , at hun foler anderledes , end hun tror . Derfor overlader hun Alt til hans Naade , foreskriver ham hverken Tid eller <b>Maal</b> . „ Sur anstiller han sig , dog er han sod , det ved jeg . " Lad os ogsaa gjore dette , saa er vi rette Kristne . Nu ser Du videre i Evangeliet , at Kristi <b>Moder</b> viser Tjenerne srn sig til Kristus og siger ikte : Gjor , hvad jeg siger , men : gjor , hvad Han siger Eder . Alene til bans Ord stal man <b>vise</b> Enhver hm , saa ledes , nt dette Marias Ord : Hvad yan siger , det giorer , er og stal vcere den hele Kristenheds daglige Ord , og fornt al Menncstclcerc stal tastes til Jorden , og at vi stal fast tro , at det ittc er Kirlens Befaling , hvad der befales

<h4>1428</h4>syger det , som er oventil , hvor Kristus er . Det vil sige , at en Kristen rent er taget bort fra denne Verden , saa at han ikke mere hyrer til dette Liv , hverken under Paven eller under Kejseren eller under nogen Skabning; men hvor Kristus er , der flat ogsaa en Kristen vcere . Men Kristus er ikke her , saaledes maa ej heller en Kristen vcere her . Derfor kan ej noget Menneske indeslutte hverken Kristus eller en Kristen i visse sceregne Regler . Det heder : Han er ikke her . Du maa ikke soge ham i de Ting , som man finder paa Jorden . Du finder ham ikke i din Fasten , Vaagen , Klceder eller noget Saadant , det er idel tomme Skal . Det heder altid : Ikte her . Det tomme Skal ifyrer han sig ikke . Som Kristus er over Alt , saaledes er en Kristen ogsaa over Alt . Kristus har ved sig selv overvundet og forladt Alt , og just derfor , fordi vi tror dette , kaldes vi ligesaa vel som han : Ikke her . Paulus siger Kol . 3 , 2 : Tragter efter det , som er oventil , ikte efter det , som er paa Jorden ; thi I ere dyde , og Eders Liv er skjult med Kristus i Gud . Vort Liv stal vcere over al menneskelig Retfcerdighed , <b>Visdom</b> og Fromhed . Saalcenge Du bliver i Dig selv , er Du ikke from . Thi en Kristen , han vcere nu i hviltmsomhelst Stand , Fyrste eller Undersaat , Herre eller Tjener , <b>Husmoder</b> eller <b>Pige</b> , saa er han over Alt , som er paa Jorden . Thi derfor kaldes han ogfaa en Kristen , fordi han bliver i den Mand , fom er dyd og opstanden og er ikte her . Det gaar saa ikke videre end ind i Hjertet og Samvittigheden . Naar Du nu saaledes har den Skat og Rigdom , som er over Alt , saa gaa ned og sig : En Kristen maa dog endnu leve og vcere blandt Menneskene . Thi Gud vil ikke ved Evangeliet ophceve det verdslige Regimente . Han vil alene regjere <b>Aand</b> og Hjerte ; disse stal mcm lade vcere uhindret af Love , og i de Stykker , fom hyrer dertil , stal hverken Kejfer eller Pave regjere ; men med Legeme og Liv er jeg en Gjcest og vil forholde mig ligesom andre Mennesker og gjyre Godt mod Verden .

<h4>1434</h4>gjelde for os , nåar vi tror paa ham , fordi at han forekommer og kalder de Kristne sine Brpdre , som troede det og saa det dog ikke ligesom Apostlene . Og han venter ikke her , indtil vi beder eller cmraaber ham derom , at vi stal blive hans Brpdre . Lad kun En tale om vor Fortjeneste! Hvad havde Apostlene fortjent ? Petrus havde tre Gange fornegtet sin Herre ; alle de andre Disiple havde flygtet fra ham . Derfor er dette Ord sendt til dem ved <b>Kvinderne</b> , lutter Madens og Barmhjertighedens Ord , som Apostlene dengang fplte godt , og vi fsler det ogsaa, naar vi sidder midt i Synd , Anfegtelse og Fordummelse . Det er et Ord fuldt af al Twst , at Kristus antager sig en snadan Synder som Dig og mig og kalder os Alle sine Brpdrc . Er nn Kristus vor Broder, saa skulde jeg nok puste at vide , hvad der mangler os . Ligesom det nu er med legemlige Brpdre , faaledes gaar det ogsaa til her . Legemlige Brpdre sidder i de samme Goder , har tillige en Fader og en Arv , ellers var de ikke Brpdre . Saaledes sidder ogsaa vi med Kristus i de samme Goder og har tillige en Fader og en Arv , hvilken Arv ikke bliver mindre ved Deling , men bliver altid stprre ; thi det er en <b>aandelig</b> Arv . Den legemlige Arv bliver mindre , naar man deler den i mange Dele ; men heri den anndeligc Arv gaar det saaledes til , at hvo , som snar en Del , han faar det Hele . Men hvad er nu Kristi Arv ? Ham tilhorer Livet og Dpden , Synden og Naaden og Alt , hvad der er i Himlen og paa Jorden , evig Sandhed og Retfærdighed . Han regjerer over Hunger og Tsrst , over Lytte og Ulykke , over Alt , hvad der kan tcentes , det vcerc sig i Himlen eller paa Jorden , ikke alene aandcligt , men ogsaa legemligt : han har Alt i sin Haand . Naar jeg nu tror paa ham , saa bliver jeg delagtig med ham i alle hans Goder og faar ikke alene en Del , men ligesom han Alt , evig Retfærdighed , evig <b>Visdom</b> og styrke , og bliver en Herre over alle Ting . Legemet skal ikke hungre ; Synden stal ikke trykke mig ; jeg stal ikke mere frygte for Dpden eller forfcerdes for Djevlen ; ingen Ting mangler mig , fordi jeg har faaet Alt i Kristus . Heraf kan man nu godt forståa de forskjellige Ord i Profeterne og Salmerne . Salm . 34 : De unge Lpuer blive nMprftige og hungre , men de , som ' spge Herren , stulle itte have Mangel paa noget Godt . Det kommer Alt deraf , at vi er og kaldes Kristi Brpdre , ikke af Fortjeneste , men af idel Naade . In , naar Gud gav os dette i Hjertet , at vi fplte det , saa var vi karske . Men det gaar ind igjennem det ene Ore og ud af det andet .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012121007148'target=_>Holm, Edvard, 1874, Verdenshistorie efter Cesare Cantù</a><h4>2319</h4>Beslutuing om at udrydde alle deres Plageaander . De brændte Borgene , dræbte Adelsmændeue , krænkede deres <b>Kvinder</b> , iførte sig paa riaragtig <b>Vis</b> deres Klæder og pralede spottende med deres Titler . En Mand stegte de og tvang hans <b>Hustru</b> og Døttre til at spise deraf . Naar man spurgte dem , hvorfor de krænkede alle menneskelige og guddommelige Love , svarede de : „ Det vide vi ikke , men vi gjøre det samme , som vi have set de andre gjøre . " „ Det var deres <b>Maal</b> , " sagde de , „ at udrydde ethvert Spor af Adelsmænd og Riddere af Jorden " . Adelen samlede sig nu fra alle Sider for at kæmpe med de uordnede Bondehobe ; deres Høvding , Charlot , blev dræbt , og Bøddelsværdet kvalte de sidste Dønninger af Opstanden i Blod .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010052806010'target=_>Scharling, C. Henrik, 1872, Menneskehed og Christendom i deres historiske Udvikling</a><h4>545</h4>Trods al den Pomp og Pragt , som her har hersket , trods den rastlsse Virksomhed , som blev udfoldet , spore vi dog Intet til Udfoldelsen af et hsiere <b>Aandsliv</b> . Historiske Efterretninger have disse Folk kun i ringe <b>Maal</b> efterladt sig , der findes kun Indskrifter og korte Beretninger om Felttog og Byggeforetagender indridsede paa bramdte Leertavler og Cylindere samt paa Billedvcerkerne i Kongepaladserne. De sidste give ligesom de tilsvarende i LEgypten et anstueligt Billed af hine Oldtidsfolks Leveviis og Sedvaner , af deres grusomme Krigsfsrelse , af deres religisse Forestillinger , men give kun ringe Kjendskab til de indre Forhold og Tilstande . Vi maae da atter her tye til fremmede Historieskrivere , iser de grcrste og jsdiske , af hvilke dog de fsrste maae benyttes med Varsomhed . Ved Siden af stor Pragt og Luxus , megen Virksomhed og Driftighed modtage vi tillige Indtrykket af Vellevnet og übunden Nydelsessyge , som tilsidst staner over i stor Usedelighet » , hvad hyppigt er Tilfelde i rige og blomstrende Handelsstader . Men her blev den tilmed bragt i ncer Forbindelse med de religisse Forestillinger , ja ligefrem betragtet som Gudsdyrkelse . Som Grund til Tilværelsen antoges et dobbelt Princift, et mandligt og et qvindeligt , det fsrste aulende , det andet fsdende , fra disse nedstamme atter de forskjellige Guder , derfor bliver Kjsnsforholdet et Udtryk for Gudsforholdet . Saaledes forteller Herodot , at i Babylon var hver <b>Qvinde</b> , ligesaa vel den fornemste som den ringeste , forpligtet til een Gang i sit Liv at bringe sin Kydskhed som Offer til Mylitte , den babylonisteKjerlighedsgudinde ^ ) . Denne Sandselighed og Appighed gaacr stundom over til et formeligt Raferi , saaledes naar Cybeles Prester i Phrygien lemlestede sig selv . Til den anden Side udarter atter denne sandselige Tummel i vild Grusomhed : ved den syriske og phsniziske Afgudsdyrkelse I denne

<h4>1278</h4>Hvor Livet antager Charakteren af Familielivet , der fslger det af sig selv , at <b>Qvinden</b> maa nyde Agt og 3 Ere , thi Familielivet bceres og opretholdes for en vcesentlig Deel af hende . Saaledes forholdt det sig ogsaa i Israel . Vistnok var Fleerkoneriet ikke forbudt i Moselov , der i saa Henseende lcempede sig efter de ssterlandste Skikke ( 5 Mos . 21 , , 5 ) , ligesom selv David havde flere Hustruer og Salomon fortabte sig i et heelt Harem , men hvad Lovens Bogstav ikke havde kunnet gjennemfsre , det formaaede dog Lovens <b>Aand</b> , thi Fleerkoneriet aftog stedse mere og mere , indtil det omsider ganske veg Pladsen for Monogamiet ( jvnfr . Mal . 2 , 14 , Sal . Orosprog 31 , 10 fig. ) . Heller ikke var ZEgtefkabet naaet til sin fulde Anerkjendelse, idet paa ssterlandsk Viis dets <b>Formaal</b> sattes ikke i sig selv , men i at avle Born . Men alligevel var det langtfra , at <b>Qvinden</b> betragtedes som en Trcelqvinde for Manden , tvertimod hun nsd Agt og LEre ved Siden af ham , kunde endog mangen Gang scrtte sin Villie igjennem mod hans ( Sarah , Rebekka ) , ligesom G . T . frembyder mangfoldige Exempler paa fromme og crdle Qvindeskikkelser ( Ruth , Hannah , Abigail o . A . ) , der vidne om , at om hun end maatte underordne sig Manden ( 1 Mos . 3 , 16 ) , saa agtedes dog hendes personlige Vcrrd for lige med dennes .

<h4>1494</h4>fri Tilslutning i Kjcerligheden . Her gjcrlder kun Personlighedens Vcerd , men denne er uendelig , efterdi Personligheden selv er Uendelig. Al Beregning ester Tal og <b>Maal</b> ophsrer , thi hver enkelt Personlighed er i sin indre Uendelighet » ligesaa meget vcrrd som alle de Andre i deres : den ene Synder , som omvender sig , volder mere Glcrde i Himmerige end de nioghalvfems Retfcerdige ( Luk . 15 , 7 ) . Derfor er det , at Christus med Kjcerlighed antager sig Bsrnene , de Smaae , de Syge og Besatte , de af Samfundet Forncrgtede og Forstudie , alle dem , som for Oldtidens Betragtning havde lidet eller intet Vcerd , medens hans kjcerlighedsfulde Vie har stuet det uendelige Personligheds Vcert » , som ogsaa disse have , og som gjsr ogsaa deres Frelse dyrebar i Guds Bine . Den sondrende Adstillelse med sine dybe Kloster , som Oldtidens Tankescet og Statsliv havde sat mellem Menneskene , forsvinder . Forskjellen mellem Mand og <b>Qvinde</b> , Fri og Trcel , Landsmand og Mlcending , der efter Oldtidens Anskuelse deelte Menneskeheden i tyende Classer , virkelige Mennesker og Skinmennesker , er her ophcrvet ( Gal , 3 , 28 , Kol . 3 , ii ) . Seete fra Evighedens Standpunkt ere alle Mennesker lige , de ere alle frie Personligheter , stabte til at elske og elskes . Selv den ringeste Menneskesjel er mere vcerd end Alverdens Sole og Stjerner og Himmelkloder , thi den er stabt i Guds Billed , det ere de ikke . Og paa samme Viis forsvinder al Forstjel mellem hsiere og ringere Aandsbegavelse mellem Menneskene indbyrdes , thi det at elske er mere end selv den hsieste Aandsbegavelse ( 1 Kor . 13 ) . ' ) — Naar derfor den humane <b>Bevidsthed</b> viser saa stor Tilbsieligheo til at anlcegge den naturlige <b>Begavelse</b> som Maalestok og at scette Genierne i en egen Rangclasse for sig som en Art Halvguder , der blive Gjenstand for en sceregen Cultus , saa vil Christendommen Intet vide heraf , den lcegger al Vcegten paa Personligheden , og Maalestokken bliver ikke den naturlige <b>Begavelse</b> , men Kjcerligheden : ikke

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112201034'target=_>Taine, Hippolyte, 1874, engelske Literaturs Historie</a><h4>1342</h4>Man kan let tænke sig , hvor stærke Karakterer , der behøves , for at bære disse voldsomme Dramaer . Alle de Optrædende ere beredte til yderlige Handlinger ; de tåge en Beslutning og Handlingen følger saa hurtigt som et Kaardestød ; ved hver Vending i Scenerne seer man deres lynende Øine , deres blegnende Læber , seer man alle deres Muskler skjælve og hele deres Legeme anspændes . Deres Overmaal af Villie faaer deres kraftige Hænder til at knytte sig , og deres Lidenskabelighed samles og bryder ud i Lyn , som sønderrive og hærge Alt omkring dem og i deres egen Sjæl . Man kjender dem , disse tragiske Helte , Jago , Kichard 111. , Lady Macbeth , Othello , Coriolan , Hotspur ; de ere Alle rigt udstyrede med <b>Aand</b> , med Mod ogAttraa , de Fleste ere afsindige eller forbryderiske , og Alle fældes 66 ved deres egen Gjerning . Og udenfor Shakespeare finde vi ligesaa mange som hos Shakespeare selv ; jeg vil fremhæve en Eneste af dem , 6 n6nu denne Gang fra Webster ; han er den , der næst efter Shakespeare har trængt længst ind i den djævelske og løsladte Naturs Dybder . The white devil , kalder han sin <b>Heltinde</b> ; hans Victoria Carambona tåger Hertug Brachiano til Elsker , og ligefra deres første Møde af , tænker hun paa <b>Maalet</b> .  For at faa Tiden til at gaa , vil jeg fortælle Deres Naade en Drøm , jeg havde

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014031808023'target=_>Luther, Martin, 1861, Dr. Martin Luthers udførlige Forklaring over Pauli Brev til Galaterne</a><h4>974</h4>de allerhelligste og kosteligste Ord , som indeholdes i den . Thi er ikke dette kostelige , trsstelige Ord : Jeg er Herren , din Gud , som udførte dig af Egyptens Land o . s . v. ' , du stal ikke have fremmede Guder for mig o . s . u . , jeg gjssr Barmhjertighed i tusinde Led o . s . v. Ligeledes: Hlrdre din Fader og din <b>Moder</b> , paa det det maa gaa dig vel , og du maa lcrnge leve i Landet ? Denne allerhsieste og stsrste guddommelige <b>Visdom</b> , som er den allerstjsnneste , bedste og kosteligste Lcere , kunne de ikke taale , men sige : Kjcrre , lad ikke Gud tale med os , at vi ikke stulle dp ; men tal du med os ! 509. Nu er det dog sandelig et over al Maade stort Under , at En ikke stal kunne hore sit eget hsieste og kosteligste Gode , nemlig at du har en Gud , og det en saadan Gud , som er naadig og barmhjertig , som vil <b>vise</b> dig Miskundhed i mange tusinde Led o . s . v. Ligeledes , at du ikke stal kunne hsre , at Gud selv vil beskytte og bevare dig , naar han siger : du stal ikke slaa ihjil ; du stal ikke bedrive Hoer ; du stal ikke stjcrle . Thi hvad gjsr vor Herre Gud med disse Bud Andet end ligesom med en Ringmur og Vcstjoermelse at beskytte og bevare dit Liv , din Mgtehustru , dit Gods og din Eiendom , mod alle onde , trodsige Misdæderes ' Vold og Angreb ? 510. Derfor kan og formaaer Loven ikke at gjsre Mere , end at gjer den med sin Glands oplyser Samvittigheden , saa at den erkjendcr sin Synd , Dsden , som den har fortjent . Guds Dom , Fiendstab og Vrede , Thi fsr Loven kommer , er jeg sikker , bekymrer mig ikke om min Synd ; men saasnart den kommer , vises og aabenbares mig Synden , Dsden , Helvede o . s . v. Dette kan dog ikke kaldes at blive ' retfoerdig , men meget mere at blive skyldig , at blive Guds Fiende , at fordommes til Dsden og Helvede . Saaledes er nu med eet Ord dette Lovens acmdelige og rette Gjerning , som den tilslut udretter , nemlig at den ikke gjsr Folk bedre eller frommere , men megetmere varre , det er , at den aabenbarer dem Synden og bringer dem til at erkjende den , paa det de ved denne Syndens Erkjendelse skulle blive ydmygede , forfærdede og ssnderknuste og saaledes lamges efter Naaden og den hsilovede Sved . Dette er Summen af det Bevis , som St . Paulus har indfsrt af det Ord Midleren .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016042848133'target=_>, 1875, Vor Herres og Frelsers Jesu Christi nye Testamente, med Davids Psalmer</a><h4>1846</h4>23. Der de sagde sig at være <b>Vise</b> , bleve de Daarer , 1 C0r.1 , 20. 23. og have forvendt den ufor ? krcrnkelige Guds Herlighed til Lig ? u hed med Billedet aset forkrænkeligt Meimeste og af Fugle og af firefodl z dede Dyr og af Orme . , d < 24. Derfor haver og Gud givet , dem hen i deres Hjerters Begjcerin ? , , ger til Ureenhed , saa at ( de ) vancere deres eg. neLegemer indbyrdes . Ap.G .14 , 16. " 25. DehaveforvendtGudsSand ? ' ' hed til Logn og have dyrket og tjent Skabningen over Skaberen , som er velsignet i Evighed , Amen ! 26. For denne Sags Skyld haver ! Gud givet dem hen tll stjcendigeLy ? , , sier ; thi baade deres <b>Qvinder</b> for , vendte den naturlige Brug til den „ unaturlige , 27. ogdesligestebleveogsaaMcen ? ! h dene , som forlove Qvindens natur ? i lige Brug , optiendte i deres Lyst til > hverandre , saa atMcend med Mcend svede Uteerlighed og fik deres Vild ? ! farelses Len , som det burde sig , paa j dem selv . I C0r.6 , 9. ! » 28. Og ligesom de ikke holdt ( det ) 3 for godt at have Guds <b>Kundskab</b> , f , saa gav Gud dem hen til et Sind , - som intet duer , saa at de gjore det t , Ussmmelige , 29. opfyldte med al Uretfærdig ? ! hed , Horeri , Skalkhed , Gjerrighed , i Ondstab , fulde afAvind , Blod ' torst , o Triette , Svig , Vanart , e 3 l ) . j3retudere , Bagvastere , Guds ?

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112101060'target=_>, 1866, Konkordiebogen, eller Den evangelisk-lutherske Kirkes Bekjendelsesskrifter</a><h4>177</h4>i sin Bog  om Hedningernes Kaldclse " ' ) og paa andre Steder . Thi saaledes siger han i sin Bog om Hedningernes Kaldelse :  Ojenlosningen ved Christi Blod vilde blive agtet ringe , og de menneskelige Gjerningers Fordringer vilde ikke blive underlagte Guds Barmhjertighet , , hvis Retfcerdiggjorelsen , som sker ved Naaden , skulde skyldes forudgangne Fortjenester , saa at den ikke blev den Givendes Gave , men den Arbejdendes Len . " 4. Men endskjsnt denne Lcere agtes ringe as de Uerfarne , saa erfare dog de gudfrygtige og cengstede Samvittigheder , at den skjenker saare megen Trost ; thi Samvittighederne kunne ikke blive beroligede ved nogen Gjerning , men alene ved Troen , naar de holde sig fuldt forvissede om , at de for Christi Skyld have en forsonet Gud , som Paulus lcerer :  Retfcerdiggjorte ved Troen , have vi Fred med Gud " Rom . 5 Hele denne Lcere maa henfsres til hin den forfcerdede Samvittigheds Kamp og kan uden denne Kamp ikke forstaaes . Derfor dsmme uerfarne og verdsligsindede Mennesker ilde om denne Sag , idet de drsmme om , at den christelige Retfcerdighed intet Andet er , end den borgerlige og filosofiske Retfcerdighed . 5. Fordum plagedes Samvittighederne med Lceren om Gjerningerne , fik Intet at hore af Evangeliets Trost . Nogle drev Samvittigheden ud i Drkenen eller md i Klostrene , idet de haabede der at fortjene Naaden ved Munkeliv . Andre udtcenkte andre Gjerninger for at fortjene Naaden og fyldestgisre for sine Synder . Derfor var det saare fornsdent paany at foredrage og fornye denne Lcere om Troen paa Christum , forat de cengstede Samvittigheder ikke skulde fattes Trost , men at de kunde vide , at ved Troen paa Christum annammes Naade , Syndsforladelse og Retfcerdiggjsrelse . 0. Ogsaa paamindes Folket om , at det Ord Tro her ikke betegner den historiske <b>Kundskab</b> alene , saadan som den stndes ogsaa hos de Ugudelige og Djcevelen , men at det betegner en Tro , som tror ikke blot Historien men ogsaa Historiens Virkning , nemlig denne Artikel , Syndernes Forladelse , at vi ved Christum have Naade , Retfcerdighed og Syndsforladelse . 7. Den , som nu ved , at han formedelst Christum har en forsonet Fader , han kjender i Sandhed Gud , han ved , at Gud har Omsorg for ham , han paakalder Ham , han er ikke uden Gud , ligesom Hedningerne . Thi Djcevlene og de Ugudelige kunne ikke tro denne Artikel , Syndernes Forladelse . Derfor hade de Gud som en Fiende , vaakalde Ham ikke , vente intet Godt af Ham . Paa samme Maade paaminder ogsaa Augustinus sine Lcesere om Ordet Tro og lcerer , at Ordet Tro i Skriften bliver taget ikke om Kundsiaben , saadan som den findes hos de Ugudelige , men om Tilliden , som troster og oprejser de forfcerdede Sjcele . 8. Desforuden lcere Vore , at det er fornsdent at gjore gode Gjerninger , ikke forat vi stulle stole paa ved dem at sortiene Naaden , men fordi Gud saa vil det . Kun ved Troen gribeS Syndsforladelse og Naade . Og efterdi den Hellig - <b>Aand</b> annammes ved Troen , blive Hjerterne fornyede og ifortc nye Tilbejeligheder , saa at de kunne frembringe gode Gjerninger . Thi saa siger Ambrosius :  Tro er den gode Viljes og den retfcerdige Gjernings <b>Moder</b> . " Uden den Hellig-Aand ere nemlig de menneskelige Krcefter fulde af ugudelige Tilbsjeligheder og for fvage til at frembringe

<h4>687</h4>Straffe ved Skjærsildens Straffe , altsaa har Nsglemagten Befaling til at eftergive en Del af Skjærsildens Straffe , altsaa frikjobe Fyldestgjsrelser fra Skjcersildens Straffe . " Hvo har lcert disse Wster denne Dialektik ? Men dette er hverken Dialektik eller Sosistik , men Sykofantik ^ ) . Derfor anfsre de dette Ord : « Omvender eder " , forat de Uerfarne , naar de hore et saadant Ord anfsrt mod os , stulle fatte den Mming , at vi afstaffe den hele Omvendelse. Ved disse Kunster forssge de at fjerne Sjcelene fra os og oppuste Hådet , at de Uerfarne samtlige stulle istemme det Skrig mod os , at saa fordervelige Kjettere , som forkaste Omvendelsen , maa udryddes . 9. Men vi haabe , at disse Bagvastelser ville Lidet udrette hos retsindige Mcend . Og Gud vil ikke lcenge taale saa stor Uforstammethed og Ondfiab. Slet sorger ogsaa den romerske Pave for sin Vcerdighed , naar han bruger faadanne Forsvarere , naar han overlader en saa vigtig Sag til disse Sofisters Dom . Thi da vi i Bekjendelsen have sammenfattet omtrent Summen af den hele christelige L < ere , stulde man til at dsmme i faa store , saa mange og faa forskjellige Sager bruge Dommere , hvis Lcere og Tro var mere prsvet , end disse Sofisters , som have strevet denne Konfutation . Og dette siulde du , Campegius " ) , som en viis Mand have ferget for , at hine ikke i faa vigtige Sager stulde stride Noget , forn enten nu eller hos Cfterkommeme stulde synes at kunne forringe den romerske Stols Anseelse . Dersom den romerske Stol mener , at det er billigt , at alle Folk stulle anerkjende den som Troens <b>Lcererinde</b> , ber den gjsre sig Umage for , at lcerde og redelige Mcend udtale sig om Religionssager . Thi hvad vil Verden dsmme naar engang vore Modstanderes Skrift vil blive bragt for Lyset ? Hvad ville Cfter- ^ kommerne domme om disse Domme , fulde af Bagvasielser ? Du fer Campegius, at dette er de sidste Tider , i hvilke Christus har forudsagt , at der vil vcere faare stor Fare for Religionen . I altsaa , der ligesom stulle staa paa Vagt og styre Religionens Anliggender , I burde i disse Tider anvende scerlig Klogstab og Omhu . Der er mange Merker , som , hvis I ikke se eder for , true det romerske Kirkevesen med en og svar ^ Forandring . Og du fejler , dersom du mener , at man kun ved Magt og Vold har at holde Kirkerne tilbage . Menneskene forlange at blive underviste om Religionen Hvor Mange , mener du , er der vel ikke ej alene i Tyskland , men ogsaa i England , Spåmen , Frankrige , Italien , ja endog i selve Byen Rom , som , fordi de se , at der er opkommet Stridigheder om de vigtigste Anliggender , begynde paa en eller anden <b>Vis</b> at tvivle og i Stilhed harmes over , at I vegre eder for rigtig at underssge og dsmme angaaende disse saa hsjvigtige Ting , at I ikke bringe de tviolende Samvittigheder til Klarhed , at I kun give Befaling til at vi stulle undertrykkes og udryddes ved Vaabenmagt . Der er mange ret ^ sindige Mcend , hvem denne Tvivl er bitrere end Dsden . Du betenker ilke noksom , hvilken hsjvigtig Sag Religionen er , dersom du mener , at det er en ringe Wngstelse , som retsindige Mcend fsle , naar de paa en eller anden Maade begynde , at tvivle om en Troens <b>Lcerdom</b> . Og denne Tviol kan ikke Andet end fsde det bitreste Had mod dem , som , medens de burde bringe Samvittighederne Legedom , tvertimod ssge ved sin Modstand at forhindre Sagens Opklarelfe . Vi tale her ikke om , at I have at frygte for Guds Dom . Thl herom , mener man , bekymre Paverne sig kun lidet , hvilke selv

<h4>871</h4>19. Paverne meddele hver Dag Dispensationer , forandre hver Dag andre , endog meget gode Love ; i denne ene Lov om Prcesternes ugifte Stand ere de jernhaarde og übsnhsrlige , stjont det er vel bekjendt , at det er en blot og bar menneskelig Lov . Og netop denne Lov stjcerpe de nu endog paa mange Maader . Kirkeloven byder at afscette Prcesterne ; disse lidet heldige Lovfortolkere afscette dem ikle fra Cmbedet , men hcenge dem op i Trceerne . Mange retsindige Mcend myrde de grusomt alene for Wgtestabets Skyld . Og just disse Myrderier <b>vise</b> , at denne Lov er  Djcevles <b>Lcerdom</b> . " Thi da Djcwelen er en Manddraber 8 , 44 ^ , forsvarer han sin Lov ved saadanne Myrderier . Vi vide , at der tages Answd af Splittelsen , idet vi synes at have revet os lss fra dem , som holdes for at vcere de ordentlige Biskoper . Men vore Samvittigheder ere ganske rolige , eftersom vi vide , at vi , om vi end af al Magt snste at oprette Enighcden , dog ikke kunne tilfredsstille vore Modstandere , medmindre vi forkaste den aabenbare Sandhed og saa forene os med dem om at ville forsvare denne uretfcerdige Lov , oplsse sluttede Wgteskaber , myrde de Prcester , der ikke ville adlyde , jage ulykkelige <b>Kvinder</b> og faderlose Born i Landflygtighed . Men da det er ganske vist , at disse Vilkaar mishage Gud , saa ville vi slet ikke vcere bedrsvede over ikke at have noget Fcellessiab med vore Modstandcre i saa mange Mord . 20. Vi have fremstillet de Grunde , hvorfor vi ikke med god Samvittighed kunne vcere enige med vore Modstandere , naar de forsvare " den pavelige Lov om bestandig ugift Stand , nemlig fordi den staar i Strid med guddommelig og naturlig Ret og afviger fra selve Kirkelovene , fordi den er overtroisk og fuld af Farer , endelig fordi det Hele er Hykleri . Thi ikke bliver denne Lov paabuden for Religionens Skyld , men for « tzerredsmmets Skyld , og for dette tåger man paa den ugudeligste Maade Religionen til Paasiud . Mod disse uomstsdelige Grunde kunne forstandige Mennesker ikke anfsre Nogetsomhelst.

<h4>978</h4>Den auzsburgfie Konfesjon ? Apologi . Art . 13. ( ssonf . 27. ) ^ tilfsjet denne Berigtigelse , at den er en Stand , hvorved man erhverver sig Fulokommenhed . Antage vi dette , saa vil MunKlivet ilke vcere Fulokom . menhedens Stand mere , end Bondens eller Haanrverkerens Lerescet . Thi det er ogsaa Stillinger , hvorved man erhverver sig Fuldtommenhed . Thi alle Mennesker bsr , Enhver i sit Kald , jage rfttr Fuldkommenhed , det er , voxe i Gucsfvygt , i Tro , > Kjccrlighed til Nasten og deslige aandelige Dyder . 13. Der findes i Historien Eneboere , der have erklceret de forskjellige Levescet ftr lige fuldkomne , som f . Er . . Antonius ' ) og Andre . Om Antonius skrives der , at , da han bad om , at Guv vilde <b>vise</b> ham , hvor langt han veo dette sit s ' evescrt havde bragt det s ^ i Fuldkommenhed ^ , blev der i Dromme vist ham en Skomager i Staden Alexandria , med hvcm han lunde sammenlignes . Dagen dcipaa lommer Antonius til Byen og gaar hen til Slomagcren for at underssge hans Gudfrygligheds Dvelser og Gaver , taler med Manden , hsrer Intet , udenat han om Morgencn med nogle faa Ord bad for den hele Stad og derpaa drev fit Haandverk med Flid . Nu sijsn » nede Antonius , at han ikte maatte tilskave det Lcv.scet , som han havde valgt sig , Netfcerdigajorelse . 14. Men skjsnt vore Modstandere nu holde mere Maade mcd sine Lovtaler over Fulekommenheden , mene de dog i Virkeligheten noget Andet . Thi de scelge Fortjenester og overfsre dem paa Andre under det Foregivende , at de iagttage baade Bud og Raad , og saaledes mene de virkeligen , at de have Fortjenester tilovers . Men hvis dette ikke er at anmasse sig Fuldkommenhed, hvad sial da laldes saa ? Derncest er det i selve Kunfutationen bestemt udtalt , at Munkene strcebe at leve mere nojagtigt ester Evangeliet . Den tilsiriver altsaa de menneskelige Overleveringer Fuldkommenhed , saafremt Munkene af den Grund leve nsjagtigere efter Evangeliet , fordi de ilke have Cjendom , foroi de ere ugifte , fordi de folge en bestemt Regel i Klccdedragt , Spiser og deslige Smaating . 15. Fremdeles siger Konfutationen , at Munkene fortjene det evige Liv i rigere <b>Maal</b> , og den anfsrer det Skriftsted :  Hver , som har forladt Hus o . s . v. " sMatth . 19 , 29 ^ . Ogsaa her tillcrgges de s.lvgjorte Gudsdyrlelser Fuldkommenhed . Men dette Skriftsted har Intct at " gjsre med Munkelivet . Thi ilke er det Christi Mening , at det at forlade Forcrldre , <b>Hustru</b> , Brsdre , er en Gjerning , som maa gjsres , fordi den fortjener Syndernes Forladelse og det evige Liv . la , forbandet er en saadan Forladen . Thi det ster til Christi Forhaanelse , naar En af den Grund f ^ rlader Forcrldre eller <b>Hustru</b> , at han ved felve denne Gjerning skulde fortjene Syndernes Forladelse og det evige Liv . Men der gives en dobbelt Forlaten . En sier uden Kald , uden Guds Befaling ; denne billiger Christus ikke Match . 15 , 9 j . Thi Gjerninger , som vi s.lv uovcelge os , ere unyttige Gudsdyrkelse . Men heraf fremgaar gansie klart , at Christus ikke derfor billiger dmne Flugt , fordi Han taler om at forlade <b>Hustru</b> og Born . Vi vide jo , at Guds Befaling forbyder at forlade <b>Hustru</b> og Born . En anden Forladen er den , som ster efter Guds Befaling , nemlig naar Magten noder os til enten at vige for Tyranner eller at fornegte Evangeliet . Her have vi den Befaling , at vi heller skulle lide Uret , heller låre vs bersve ikke alene Formue , Wgtefcelle , Born , men endog Livet . Denne Forladen billiger ChristuZ , og derfor lcegger Han til  for Evangeliets Skyld " , for at betegne , at Han taler om dem , som ikke gjore <b>Hustru</b> og Born Uret , men som for Evangeliets Skyld lide Uret .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015080627001'target=_>Landstad, M.B., 1875, Kirkesalmebog</a><h4>4082</h4>bevar Kongen og Landet med al kirkelig oq verdslig øvrighed fra al Ulykke . Gud give os Fred og Rolighed . Gud unde os Helbred og Sundhed . Gud naadigen velsigne vor ( Ager og Eng ) Mad og Drikke , vor Kurv og vore Levninger . O Herre , tank i Naade paa alle Syge og dem , som lide Angest og Pine . Giv dem en god Nat , send Lindring og Lcrgedom , send dem TrM og Haab . Hjcrlp dem til at kunne vare glade i Haabet , taalmodige i Trangselen og varagtige i BMnen , og lad ' deres Forløsningstime, som bie paa dig , ikke uccre langt borte . O Herre , var mig arme Synder ( min Husbonde , <b>Hustru</b> , Foraldre , BMn ) min Slagt og Venner fjern og nar og alle Mennesker naadig for Jesu Kristi Skyld ! Vogt os med Hus og Hjem , Gods og Formue fra Ilds-NFd , Vands- Vaade , Tyvehaand og alt andet Ondt , og lad ingen Ulykke komme inden D ^ re . Der lagges Dage , Uger og Aar til min Alder . Vor Klokke slaar , vor Tid bortgaar — og Evigheden forestaar . O Herre , lar mig saa at talle mine Dage , at jeg maa bekomme <b>Visdom</b> i Hjertet! Lar mig saaledes at bruge de dyre Naade- Timer , at jeg altid med Frimodighed kan bie efter min Fraskilleises Stund og forvente i Haabet Guds Bsrns herlige Frihed . Ak , Herre Jesu , kom , kom naadig , kom i en god Stund ! I din Haand befaler jeg bande Legem og <b>Aand</b> , du forliste mig , Herre , du trofaste Gud . Amen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111103019'target=_>, 1856, Vor Herres og Frelsers Jesu Christi nye Testamente</a><h4>1169</h4>ket deres Stad var bygget , for at styrte ham ned . 3 l ) . Men . han kom igjenncm nridt imellem dem og mk bott . Joh . 8,59 . 111. 31. Og hau kom nev til Capernaum , en i Galilea, og lcerte vern paa Sabbaterne. Matth . 4,13 . 32. Og de forundrede sig saare over hans <b>Lcerdom</b> ; thi hans Tale var med Myndighed . Matth . 7,28 . 29. Mare . 1,22 . 33. Og der var et Menneske i Synagogen , som havde en ureen Djoevels <b>Aand</b> , og raabte med hoi Nost Mare . 1,23 fg . 34. og sagde : Eja ! hvad have vi med dig at skaffe , Jesu af Nazareth ? Er du kommen for at fordcerve os ? Jeg kjcnder dig , hvo du er , den Guds Hellige . 35. Og sesus truede ham og sagde : Ti og far ud af ham ! Og Djcevelen kastede ham midt iblandt dem , og foer ud af ham og gjorde ham ingen Skade . 36. Og der kom en Rcedsel over Alle , og de taledc med hverandre og sagde : Hvad er dog dette , " at han byder de urene Aander med Myndighed og Magt , og de fare ud ? 37. Og Rygtet om ham udspredtcs allevegne i det omkringliggende Land . 38. Men han stod op og gik fra Synagogen ind i Simons Huus ; og Simons Hustrues <b>Moder</b> var plaget med en svar Feber , og de

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009080603010'target=_>Hagenbach, Karl Rudolf, 1860, nyeste Tids Kirkehistorie</a><h4>58</h4>det 18 de og tillige ogsaa det 19 de Aarhundredes Kirke , maa ikke tanke sig den indcsluttet af Pale og Mure , som hlndre Blikket udadtil , han maa heller ikke sparre sig Adgangen ved at nedramme saadanne Pale ; ellers <b>moder</b> han rigtignok en jaNimerlig Ruin med meget forfaldne Mure , han vandrer paa en Kirkegaard , der virkelig kun er en Kirkegaard fuld af Naaddenhed og Dodningcbecn . hvorpaa kun hist og her et Kors og et Monument med udslettet Indstrift rager frem blandt det frodlge Ukrudt ; men han maa have sit Blik friere op. Udover de nedrevne Mure og Ruiner , udover Doduingebenene, der ligge ssrgeligt nok fpredte omkring og vel kunne afpresie ham et Suk af og til , maa han med prophetlsk <b>Aand</b> ogfaa vende sig til de Templer , paa hvilke Verdensaanden narmest synes at bygge , og som ved forste Diekast nasten fee mere hedenske end christelige ud , som mere minde om det livlige grafte Vasen , end om Middelalderens gothiste Former ; han maa ikke blot lytte til de Stemmer , der lyde fra Kirken , men ogfaa til Digtcrnes og Philosophernes Roster . tll deres Noster , der — i god og i ond Betydning — oplyse og forbedre Verden ; selv Theatret , der forresten er faa langt fjernet fra Kirken , vinder nn Betydning for ham , som gribende ind i Tidens <b>Dannelse</b> . Han maa ogsaa betrade de Steder , der , om de end . Intet mindre ligne end en Kirkegaard , alligevel hjalpe med til at danne den store vide Forgaard , der efter Guds Hcnsigter skulde vare bestemt tll at omstutte Herrens Kirke ien friere Stiil . Hvo der vil forstaae det 18 de og 19 de Aarhundredes Religion og Theologie , maa idetmindste tjcnde saameget af denne Tids Philosophie med dens Kampe , at de theologiske Livssporgsmaal ikke komme til at svave i Luften for ham ; og hvo der vil vide , hvorledes det Guddommelige er bleven opfattet og vurderet af Menneskene, han maa fremfor Alt kjende det Menneskellge . saaledes som dette udprager sig i Kunst , i Sader og Skikke , saaledes som det gjennem Opdragelsen har banet sig Vei til Ungdommen , og gjennem populalre Skrifter til Folket .

<h4>1126</h4>Fra den christellge Erkjendclses Side staaer Gothe unegtelig hsiere end Schiller , forsaavidt han overhovedet legger en mere omfattende , af Skolefordomme mlndre omtaaget <b>Aand</b> for Dagen . Han kunde f . Ex . langt mere uhildet bedomme den bibelske Historie og de christelige Dogmer , end den ofte lldenstabelig bevegede Schiller . Og hvorledes kunde det vere muligt Andet , end at den Mand , uuder hvis Hender Alt vandt levende Slittelfe , ikke ogsaa skulde optage Christendommens verdenshistoriste Phenomen i denne Dannelsesproces og forarbeide den i sit store Verdensbillede ? " ) Hvilken Rigdom af dybe christelige Anskuelser <b>moder</b> os ikke i Faust ? hvilken Sands for det christelige Livs fine Nuanceringer i en stjsn <b>Sjels</b> Bekjendelser ! Selv Christendommens Opfattelfe paa den hemhutlste Maade fiuder han berettiget paa sit Sted , som hans Omgang med den aandrige Froken von Klettenberg beviser . Hvilke sunde , rigtige Domme om Bibelens hoie Verd og dens opdragende Betydning mode vi itte i hans Levnetsbeskrivelse , og tlldeels ogsaa i de inostrsede Bemerkninger til hans Farvelere og navnlig i Samtalerne med Eckermann? Eller kan man , for blot at anfsre eet Exempel , sette et kraftigere Ord imod Bibelforagteme , end hvad Gothe siger : 2 > ) „ Io mere Aarhundrcdeme Mage i <b>Dannelse</b> , desto mere vil Bibelen kunne blive benyttet deels som Opdragelsens Fuudament , deels som dens Redstab , rigtignok ikke af nesvise , men af i Sandhed <b>vise</b> Mennesker . " Og saaledes endnu flere lignende Vitringer . Ja , det er vist , den nyere christetige Apologetik kunde hente langt flere Beviissteder af Gothes Skrifter end af Schillers , hos hvem nesten allevegne et Kjetteri stikker frem .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010052806032'target=_>, 1868, Konkordiebogen, eller Den evangelisk-lutherske Kirkes Bekjendelsesskrifter</a><h4>359</h4>i sin Bog  om Hedniugernes Kaldelse " ' ) og paa andre Steder . Thi saaledes siger han i sin Bog om Hedningernes Kaldelse :  Gjenlosn ingen ved Christi Blod vilde blive agtet ringe , og de menneskelige Gjerningers Fordringer vilde ikke blive underlagte Guds Barmhjertighed , hvis Retfcerdiggjorelsen , som sker ved Naaden , skulde skyldes forudgangne Fortjenester , saa at den ikke blev den Givendes Gave , men den Arbejdendes Len . " 4. Men endstjont denne Lcere agtes ringe as de Uerfarne , saa erfare dog de gudfrygtige og cengstede Samvittigheder , at den skjenker saare megen Trost ; fhi Samvittighederne kunne ikke blive beroligede ved nogen Gjerning , men alene ved Troen , naar de holde sig fuldt forvissede om , at de for Christi Skyld have en forsonet Gud , som Paulus lcerer :  Retfcerdiggjorte ved Troen , have vi Fred med Gud " Rom . 5 Hele denne Lcere maa henfores til hin den forfcerdede Samvittigheds Kamp og kan uden denne Kamp ikke forstaaes . Derfor domme uerfarne og verdsligsindede Mennesker ilde om denne Sag , idet de drsmme om , at den christelige Retfcerdighed intet Andet er , end den borgerlige og filosofiske Retfcerdighed . 5. Fordum plagedes Samvittighederne med Lceren om Gjerningerne , stk lutet at hore af Evangeliets Trost . Nogle drev Samvittigheden ud i Drkenen eller ind i Mostrene , idet de haabede der at fortjene Naaden ved Munkeliv . Andre udtcenkte andre Gjerninger for at fortjene Naaden og fyldestgiore for sine Synder . Derfor var det saare fornsdent paany at foredrage og fornye denne Lcere om Troen paa Christum , forat de cengstede Samvittigheder ikke skulde fattes Trost , men at de kunde vide , at ved Troen paa Christum annammes Naade , Syndsforladelse og Retfcerdiggjorelse . 0. Ogsaa paamindes Folket om , at det Ord Tro her ikke betegner den historiske Kundstab alene , saadan som den findes ogsaa hos de Ugudelige og Djcevelen , men at det betegner en Tro , som tror ikke blot Historien men ogsaa Historiens Virkning , nemlig denne Artikel , Syndernes Forladelse , at vi ved Christum have Naade , Retfcerdighed og Syndsforladelse . 7. Den , som nu ved , at han formedelst Christum har en forsonet Fader , han hender i Sandhed Gud , han ved , at Gud har Omsorg for ham , han paakalder Ham , han er ikke uden Gud , ligesom Hedmngerne . Thi Djcevlene og de Ugudelige kunne ikke tro denne Artikel , Syndernes Forladelse . Derfor hade de Gud som en Fiende , paakalde Ham ikke , vente intet Godt af Ham . Paa samme Maade paaminder ogsaa Augustinus sine Lcesere om Ordet Tro og lcerer , at Ordet Tro i Skriften bliver taget ikke om <b>Kundskaben</b> , saadan som den ftndes hos de Ugudelige , men om Tilliden , som troster og oprejser de forfcerdede Sjcele . 8. Desforuden lcere Vore , at det er fornsdent at gjsre gode Gjerninger , ikke forat vi skulle stole paa ved dem at fortjene Naaden , men fordi Gud saa vil det . Kun ved Troen gribes Syndsforladelfe og Naade . Og efterdt den Hellig - <b>Aand</b> annammes ved Troen , blive Hjerterne fornyede og iforte nye Tilbojeligheder , saa at de kunne frembringe gode Gjerninger . Thi saa siger Ambrosius :  Tro er den gode Viljes og den retfcerdige Gjernings <b>Moder</b> . " Uden den Hellig-Aand ere nemlig de menneskelige Krcefter fulde af ugudelige Tilbojeligheder og for svage til at frembringe

<h4>815</h4>Straffe ved Skjærsildens Straffe , altsaa har Noglemagten Befaling til at eftergive en Del af Skjærsildens Straffe , altsaa frikjsbe Fyldestgjsrelser fra Skjcersildens Straffe . " Hvo har lert disse Wster denne Dialektikk Men dette er hverken Dialektik eller Sofistik , men Sykofantik ^ ) . Derfor anfsre de dette Ord : « Omvender eder " , forat de Uerfame , naar de hore et faadant Ord anfsrt mod os , stulle fatte den Mening , at vi afstaffe den hele Omvendelse. Ved disse Kunster forssge de at fjerne Sjcelene fra os og oppuste Hådet , at de Uerfame samtlige stulle istemme det Skrig mod os , at saa fordervelige Kjettere , som forkaste Omvendelsen , maa udryddes . 9. Men vi haabe , at disse Bagvaslelser ville Lidet udrette hos retsind.ige Mcend . Og Gud vil ikke lenge taale saa stor Uforskammet ! ) ed og Ondstab. Slet sorger ogsaa den romerske Pave for sin Verdighed , naar han bruger faadanne Forsvarere , naar han overlader en saa vigtig Sag til disse Sofisters Dom . Thi da vi i Bekjendelsen have sammenfattet omtrent Summen af den hele christelige Lere , skulde man til at demme i faa store , faa mange og saa forskjellige Sager bruge Dommere , hvis Loere og Tro var mere provet , end disse Sofisters , som have strevet denne Konfutation . Og dette skulde du , Campegius " ) , som en viis Mand have sorget for , at hine ikke i faa vigtige Sager skulde skrive Noget , som enten nu eller hos Efterkommerne skulde synes at kunne forringe den romerske Stols Anseelse . Dersom den romerske Stol mener , at det er billigt , at alle Folk stulle anerkjende den som Troens <b>Lcererinde</b> , bor den gjore sig Umage for , at lcerde og redelige Mcend udtale sig om Religionssager . Thi hvad vil Verden domme , naar engang vore Modstanderes Skrift vil blive bragt for Lyset ? Hvad ville Efterkommerne domme om disse Domme , fulde af Bagvastelser ? Du ser , Campegius, at dette er de sidste Tider , i hvilke Christus har forudsagt , at der vil vere saare stor Fare for Religionen . I altsaa , der ligesom stulle staa paa Vagt og styre Religionens Anliggender , I burde i disse Tider anvende serlig Klogstav og Omhu . Der er mange Merker , som , hvis I ikke se eder for , true det romerske Kirkevesen med en og svar ^ Forandring . Og du fejler , dersom du mener , at man kun ved Magt og Vold har at holde Kirkerne tilbage . Menneskene forlange at blive underviste om Religionen . Hvor Mange , mener du , er der vel ikke ej alene i Tyskland , men ogsaa i England , Spåmen , Frankrige , Italien , ja endog i selve Byen Rom , som , fordi de se , at der er opkommet Stridigheder om de vigtigste Anliggender , begynde paa en eller anden <b>Vis</b> at tvivle og i Stilhed harmes over , at I vegre eder for rigtig at undersoge og domme angaaende disse saa hojvigtige Ting , at I ikke bringe de tviolende Samvittigheder til Klarhed , at I kun give Befaling til , at vi stulle undertrykkes og udryddes ved Vaabenmagt . Der er mange retsindige Mend , hvem denne Tvivl er bitrere end Doden . Du betenker ilke noksom , hvilken hojvigtig Sag Religionen er , dersom du mener , at det er en ringe Wngstelse , som retsindige Mend fole , naar de paa en eller anden Maade begynde at tvivle om en Troens <b>Lerdom</b> . Og denne Tviol kan ikke Andet end fode det bitreste Had mod dem , som , medens de burde bringe Samvittighederne Legedom , tvertimod soge ved sin Modstand at forhindre Sagens OpNarelse . Vi tale her ikke om , at I have at frygte for Guds Dom . Thi herom , mener man , bekymre Paverne sig kun lidet , hvilke selv

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112301019'target=_>, 1859, Norsk lovsamling for Aarene 1667 til 1858</a><h4>9740</h4>nogen leverer noget til Kirken , da ssal det dog i Regnssabet ej videre godtgisris , end for det billige Kisb . som andenstads havis kand . . « , . . . 59. Store Bvaninqers Fortingninger stulle ssee med og efter Ktrkens Forsvars Raad oq ssal al Nrbejds og Varers Fortingninq ssee i Provstens Prcrstens og nogle beste Soqnemcends Overvcrrelse . Flikteriet fortinqis i Prastens Overvarelse , naar Daqen lcrnqst er , vaa Arbejdernis egen Kost nsjeste Kisb mueligt er , og for en <b>vis</b> Penqe . oq ej Drikkevenqe , Dl eller andet tilsigis ; Og ssal saa fortingis . at saa fremt arbejdet ej er giort til Gavns , da Embeds Folket paa deris egen Bekostning ssulle det reparere . Naar de betalis , da tagis deris Bevis derpaa , som arbejdet have , eller og Prcestens med deris Segl , eller Marke , om de selv ej ssrive kunde . 60 Kirkens Bonder stulle , saa vit de Pligtige ere , gisre Rejser for Kirken : De andre Rejser stulle indforis ved Dag og vnwm , og hvad Materialier og Bygningsmcrnd de have fort til og fra Arbejoet ; Men dersom Pygningsmcendene for deris eget Wrindis ssuld drage hiem , imidlertid de arbejde , da bor Kirken det ej at betale ; Ellers om nogen stor Nep » ! - » ti « n bebovis vaa Kirkerne , da stal menige Almue uden Forssiel bielve dertil udi hvis Maade de bsr . med Arbejde , Kiorsel og andet efter gammel Scedvane . ' « > < « ' « » . e » yl . Hvis over Bygningen til overs bliver , stal , saa vtt Kirken ikke med fsrste bebsver . salgis , eller om det ikke ssee kand , sattis i god Forvaring og feris , Knkens Beholdning oq Inventniium ved Vagt , <b>Maal</b> og Maade , verre sig Materiawr , som til Bygning tiene , Baller , Bytter , Tru , Stiger , Spader , Skovle , Stillings-Tsmmer , oa alt saadant hvis ncevnis kand . . « > . < , , 62. Kirkevcergerne i Kisbstaderne ssulle holde deris Klrker as Knkernis egne Indkomme ved lige uden andre Kirkers Tillceg og Besvcering , saa og forssrge deris Soanevraster . Capellaner og andre Kirketienere med beqvemme og smkellge Boliger , oa af Kirkernis Middel holde dem ved lige og bygge derpaa hvad der fattis til For » nsdenhed . — « Juni 18 , U § 5 . 64. Tiendeboden paa Landet ssal for Kirkens Penge af Kirkevcergerne ved lige boldis . — Jfr . 2 - 22 — 1. « 5. I Kirken maa ej indleggis nogen Lading eller Vare . men Kirkerne bllve til den Brug , de ere forordnede , saa at Prasterne kunde have Platz at forrette deris Embede . oq Sognefolket maa have Rum at hore Prcediken ; <b>Mor</b> nogen herimod , have Varerne forbrudt til Kirken . - vist 23 Mrs , 1830. Grl . § 55. Frd , 21 Mrs . 1705 § 1 ? Nes , 18 Oct 1743 Frd , 22 Fet , 1805. 66. Kirkens Fortog maa ingen bruge . indhcegne , eller Bygning paascrtte uden Kirkens Forsvars Bevilling , og Kirken faar sin Rettighet » der af . — Frd , 13 Ang . 1734 § 7. Res . 14 Juni 1687 § 4 . Res 3 Mai 1704. 67. Kirkens Beholdning og indkrcevet Restanh ssal scettis paa Rente , om den bos visse Folk kand blive udsat . og naar Summen belsber sig til tyve Daler , da ssal taqis af hver tyve Daler een . o > i af bvert hundrede fem : Hvis Beholdningen er nnqere end tyve Daler , eller og den ikke til . visse Folk kand blive udsat , kand Kirkens Forsvar betro Kirkevcergerne dem . om de ere vederheftige , eller og indlegge dem paa et sikkert Stced , til Kirken dem bebsver , eller de kunde blive <b>vis</b> udsatte , og ellers maa Kirkevcergernc ikke have Kirkens Penge uden Rente . — Jfr . L. om Rentefoten 12 Oct , 1857. 68. Satte adskillige Kirker en samlet Sum Penge paa Rente , da tåger hver Kirke sin Part af Renten , og Haandstriften bliver hos den Kirke , som stsrste Part haver i Hovcdstolen . eller hvor Kirkernis Forsvar eragter den sikkerst at " W ^ ^ Naar Kirkens Regnstab hsris . da ssulle de , som det bere , vel tåge i Agt , om Kirkens Penge staa hos visse Folk , hvad Forvaring mand derfor haver , om Renten er udkommen , eller resterer , om nogen uvis Gield der iblant findts , om Renten , eller Hovedstolen , eller anden Indkomme , over rette Tid forholdis , at det Knken M beste kand indkravis . og i Tide opssrivis , hvor Uvederhaftighed ventis ; Og stulle Knkevargerne saadan Beholdning og Reitantz med lovlig kraces og Dom . det sorste mueliqt er . indkrave . og derudi af Kongens Befalingsmand og Foqderne fortnelviS . 70. Kirker , som have laant Penge af andre Kirker til deris Fornsdenhed . stulle give Renter deraf . dersom de selv ere formuende , eller have Penge Paa Rente . 71. Naar Kirkens Reqnstabs Bog er fuldstreven . da bestilles en ny paa Kirkens Bekostning , og den gamle benleggis i ' Stigts Kisten forvarligen at glemnus ; Dog ssal den tilstillis den . som Kirken ' haver i Forsvar og under Tilsyn , naar hand den begarer . som og siden den igien i Stigts-Kisten stal indlevere . . 72. Prasterne ssulle i Superintendentens og Provstens Visitatz glve tllnende , om noget af alt det , som ber befalet er , forssmmis .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081903025'target=_>Nissen, Johan, 1863, J. Nissens Samtaler over Den bibelske Historie</a><h4>947</h4>der gaae bag efter . Saadanne og lignende Betragtninger over den Dsde stulle I anstille , nåar I fslge En til Jorden ; det er bedre end at tale med Eders Sidemand om Vind og Veir og Vynyt . 2 ) Ten bringer os til at tamle Paa de Paarorcnde . Vore Tanker gaae fra den Dsde hen til de Efterlevende . Her see vi en <b>Moder</b> , en Enke fslge sin eneste Ssn til Jorden , ham , som hun maastee havde sat alt sit Haab til , som stnlde pleie hende i hendes Alderdom . Men : Mine Tanker ere ikke Eders Tanker og — . Saaledes gnaer det endnu . Vprueue miste deres Forssrger eller deres tjcerlige og smme <b>Moder</b> . Hvilke Sorger opstaae da , spsrger jeg Eder , som allerede kjende noget til Livet ? Men Guds Ord siger : Verer ikke bekymrede ; laster al Eders Sorg Pan ham . Det kjender altsaa ikke til Bekymring , men kun til Bon og Arbeide . Og Christus siger eudnu til os : Gret » ikke . Vistnol opvekker hau ilke den elskede Afdsde , men han opvekker Mod og Tillid i et forsagt Hjerte ; uventet , fsrend vi aue det , kommer Hjelpen , som her til denne <b>Moder</b> ; han opvekker Mennesker , som tilbyde deres Bistand ; han fsier det saa , at Fortjenesten og Neringen ikke standser . Kjende I Exempler herpaa fra vor By , saa forteller dem . — — Godt ; har Gnd hjnlpet saaledes eengang , hundrede Gange , saa kan han gjsre det endnu . Trsster Eder selv og Andre hermed , nåar I gaae bag efter Kisten og ere bekymrede for de Efterlevende . Med hvilke Tanker altsaa ? 3 ) Ten leder Tanken hen paa os selv . Nn gaaer jeg bag ester tenne Kiste ; e ' ngang , maastee snart , vil man gaae bag ester min. Saaledes blive Tankerne ved en Ligbegengelse alvorligere , idet de henvendes paa os selv . Jeg har fortalt Eder om Jairi tolvaarige Datter . Hvilke Tanker kunde der da opstaae hos mange af Eder ? Jeg er i samme Alder som hnn ; hvad , om jeg nu dsde ? Jesus sammenlignede Dsden med en Ssvn ; ja , den Tid kommer , da mnn stal rede Eder en Seng i Jorden , „ fire Fjele er den Pragt , hvori da jeg bliver lagt , med et Lagen og lidt meer , eier ikke saa en Fjer . " Det er ssrgclige Tanker , som Mange ere bange for ; derfor ville de ikke gjerne see et Lig eller fslge Nogen til Jorden . Nei , ikke saaledes . Den , der stal lcempe mod en Fjende , bereder sig dertil itide . Hvorledes stal man dn berede sig i dette Tilfelde ? Ved at tenke pnn Dsden , Lnder os dse , fsrend vi dse , nt vi ikke stulle dse , unar Dsden kommer . Forklar mig dette . Mange Folk bede en Bsn , nåar de gaae ind i et Sorgehuns , eller nåar de see et Lig . Hvorfor ? Fordi Dsden vekker Alvor i <b>Sjelen</b> , saa at den gaaer i sig selv , vender sig til Gud o . s . v. Hvilken Bsn bede de da ? Godt . Jeg stal lere Eder to andre korte Vsnner , som ere passende under saadanne Omstendigheder . Ps . 39 , 5. 6 : Herre , lad mig kjende mit Endeligt og mine Dages <b>Maal</b> — . Ps . 90 , 12 : Ler os at telle vore Dage , at vi faae Viisdom i Hjertet . Jeg stal fortelle Eder , hvorledes man bnr sig nd for to til trehundrede Aar siden . Nicolnns Amsdorf , som var en Ven af Luther , havde altid sin Ligtiste staacnde foran sin Seng og brugte den som en Benk . En gammel <b>Kone</b> lod sin Ligkiste gjore ti Aar fsr sin Dsd og sov mange Gange , til Middag deri . Man valgte sig en Ligtext ; havde sin Ligdragt lenge forlid ferdig ; i Ringen , som man bar paa Fingeren , stod memento mori , d . e . hust Pan , at Du stal dse o . s . v. Hvad skulde alt dette sige ?

<h4>1672</h4>1 ) Paulus for sin Omvendelse . Scmlns havde allerede i lernsalem forfulgt de Christne . Hau siger om sig selv , C . 22 , 4 : Jeg bandt og overantvordede i Fengsler baade Mcend og Kvinter ; V . 19 : Jeg hudflettede i Synagogene dem , som troede paa Dig ; C . 26 , 10 : Naar de bleve tagne af Dage , gav jeg min Stemme dertil . Hans Naferi mod Christendommen sammenlignes i V . 1 med et vildt Dyrs Fnysen . Han meente , C . 26 , 9 , at han burde gjsre Meget imod Jesu Navn . I 1 Tim . 1 , 13 siger han om sig selv : Jeg var tilforn en Bespotter og en Forfslger og en Forhacmer . Derfor var Forfslgelsen i lernsalem ham ikke nok . Han lod sig give Breve til Synagogene i Dcunascns i Syrien , V . 2 , paa det , om han fandt nogle Ment » eller <b>Kvinder</b> , som vare af denne Lerevei ( d . e . som fulgte denue Vei til Salighed , denne Religion ) , han da kunde fsre dem bundne til Jerusalem ; og hau var allerede uer ved sit <b>Maal</b> , udeu at haus Sivd havde forandret sig . Hvorledes knnde han dog gjsre alt dette ? 1 Tim . 1 , 13 : Jeg gjorde det uvitterligt i Vantro . Uvitenheten er altsaa tilregnelig , som Tilsetningen : i Vantro , ogsaa viser . Men selv om en forskyldt Uvitenhet » er Aarsag til Synden , saa er dog Redning endnu mulig ; og uaar den ikke kom fra Panlns , fra hvem kunde den da alene komme ? Fra Herren ; og fra ham kom den ogfaa ; 1 Tim . 1 , 16 : lefus Christus vilde paa mig , som den stsrste ( Synder ) , <b>vise</b> al Langmodighed, dem til et Exempel , som skulde troe pna ham til det evige Liv . Denue Begivenhet » angnner nltsnn os ; og hvorfor ? Her ville vi ogfnn anfsre Ordene i Nom . 2 , 4 : Foragter Du hnus Godheds og Tnnlmodigheds og Lnngmcdighcds Nigdcm eg veed ille , nt Guds Godhet » leder Dig til Omvcntclsc ? Hvorledes see vi denne Tnnlmodigheds „ Nigdom " i Pnnli Liv ? og hvorledes erfnre vi selv denne Rigdom ? Nnnr vi snn ofte lyve eg hevne os og fsre ngudelig og stamm elig Tale , og han san dog lader os « straffete . Hvorfor steer ta dette ? Fordi hans Godhet » vil lede os til Omvendelfe . — Da nu Paulus gaacr videre og videre paa den betraadte Bane , saa forbarmer Gud sig over ham . Han hsrte en Rsst , der sagde ( i det hebraiske Sprog , C . 26 , 14 ) : Snul , Sanl - ; og tillige saae han Herren , der strax gav sig tilkjende for ham scm Jesns . Dette fcrtellrr Bnrnnbas Apcstlene i V . 27. Ogsnn senere hnr Pnnlns seet Herren , C . 22 , 17 : Det stete mig , der jeg kcm tilbage til Jerusalem og bad i Templet , at jeg faldt i Henrykkelse og saae ham . C . 23 , 11 : Men Natten derefter stod Herren for ham og fagde o . f . v. Derfor kunde Panlus ogfaa sige , 1 Con 9 , 1 : Er jeg ikke eu Apostel ? Har jeg ikke seet vor Herre lesum Christum ? 1 Cor . 15 , 9 : Sidst af Alle blev hau seet af mig , forn det utidige Foster . Han mener her Aabenbarelsen ved Drunascns . Endnu eu Aabeubarelse af Herren ( man veed ikke , hvornaar den er

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015061048067'target=_>Hase, Karl, 1837, Kirkehistorie</a><h4>1843</h4>Mystikens Opposition mod ScholaMen aabenbarede sig fornemligen i Forfolgelsen mod Abcelard , der , endskjondt Glcede og Sorg i et meget bcvceget Liv ikke skjcenkede ham No til med videnskabelig Dybde og Besindighed at fuldfors en Omdannelse af Theologien , dog mere glimrende i Skolen og med friere <b>Aand</b> end nogensomhelst Echolastik r , tydeligen betegnede Scholastikens yderste <b>Maal</b> i d.n Grundscrlning , at man kun burde troe , hvad man forst havde indseer . Scholastikens indre Modsigelse, nemlig Kirkelcerens übetingede Auctoritet , som Abcelard heller ikke vilde overskride , ved Siden af Fornuftens Eneherredomme, traadte derved tydelig frem . Petrus Abcelard var fsdt i Palais i Bretagne af cedel Familie A . 1079. Hans Ungdomskraft fandt Glcede i , ved dialektisk Fcrrdighed at overvinde hiin Tids beromteste Lcerere . Fra 1115 holdt han paa den h . Genovefas Bjerg Forelcesninger som den meest yndede Lcerer i Philosophie og Theologie i Paris . Her gjorde hans Kjcerlighed til Heloiie ham lykkelig og ulyksalig . Hendes hoie <b>Aand</b> lod hende forsmaae at kaldes Abce ' ards <b>Kone</b> , fordi hun vilde see den elskede Mand staae , hvor det tilkom ham , nemlig blandt Kirkens Hoveder . Hendes Slagtningers skjcendige Udaad havde til Hensigt ogsaa at tilintetgjort dette . I Undseelse og ' Fortvivlelse flygtede Abcelard fra Verden til Klosteret St . Denys og blev Munk Heloise tog Sloret . Men da den akademiske Ungdoms Bonner atter havde fort ham tilbage paa hans Lcrrestol, forenede Scholastikernes Skinsyge sig med Mystikernes Had , og paa Synoden i So is so ns si 121 ) blev under Forscede af en Legat hans Indledning til Tbeologien , som ikke rettroende Fremstilling af den h . Trinitet , fordomt til at brcendes , og han selv , uhort , til afsides Klosterfængsel . Men den offentlige Deeltagelse fornyedes ved denne Mishandling , Legaten tillod , at han vendte tilbage til Klostret St . Denys . Forfulgt af Munkene , fordi han havde opdaget , at Dionys af Paris ikke var Areopagiten, flygtede han til Drken ved Nogent , hans Tilhoreres Skarer fulgde ham , levede af Skovens Frugter , og byggede hinanden Hntter , ham et Kloster , som han helligede Aanden , , Trosteren. Men paany truet med Forfolgelse , overgav han Abbediet Para c le tus til sin Heloise , og tog mod en Vocation til Ruits i Bretagne Efter et Decennium fuldt af unyttige Kampe for , som Stedets Abbed , at gjenoprette Klostertugten,

<h4>2290</h4>Den Skik at aftcelle et vist ) lntal Venner efter en Karvestok sindes allerede i det 4. Aarh . blandt Eneboere i Thebais , og undertiden tt ) selv blandt Hedningerne . Denne Nonnens Gjerningshellighed sik en bestemt Form og fremherskende Henfo > else til Guds <b>Moder</b> i Rosenkran d sen slinsariuni , der i det 13. Aarh . blev almindelig iscer ved Dominicanerne , Gudstjenesten concentrerede sig i Messen , der i sin Idee var den hellige Dyrkelse af det tilstedeværende Gudmenneske , i Virkeligheden paa mangfoldig Viis var udartet til blot udvortes Vcerk af . Overtroen , som Vantroen ofte bod tilfals . Dog blev ogsaa Kirkens Lcerecmbede iscer bestyret af vandrende Tiggermunke , og ofte , rummede kun Gudstemplet i det Frie Tilhorernes Mcengde . Mange Prcedikener ere fulde af scholastist <b>Lcerdom</b> eller phantastist Legevcerk ; > Astadsprcedikanter priste deres Varer med taabelige Fabler ; men der fremstod ogsaa Prcester , som forkyndte Guds Ord paa en folkelig Maade . Saaledes Dominicaneren Johan af Vicenza sI23H , der med vidunderlig Veltalenhed blev en Fredsstifter fcr det sonderrevne Italien , indtil han ved at indlade sig paa U » dervcerker og Politik tabte Folkets saaledes Berthold fra Regensburg sdod der rystede vilde Gemytter, bekcempede Astadsprcedikanterne og drev paa en Gudsdyrkelse i <b>Aand</b> og Sandhede ) ; saaledes ogsaa Galler af Kaise rsber g ftod der med Idealets Inderlighed i Hjertet

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100803022'target=_>Kennedy, Grace, 1850, Miss Grace Kennedy's samtlige Værker</a><h4>43</h4>Allerede tidlig udmcrrkede hun sig ved Forstningsaand og lagttagelsesevne . Hun leste meget gjerne , lcerte Alt , hvad der blev buden hendes <b>Aand</b> , med en overordentlig Hurtighet » , og tilegnede sig med Lethed de <b>Kundskaber</b> og Fcerdigheder , der i hendes Stand i Almindelighed udgjore Bestanddelene af den <b>qvindelige</b> <b>Dannelse</b> , atene med Undtagelse af Musik , som hun aldrig lagde sig ester . Et seregent Talent besad hun for Tegning , hvormed hun til forskjellige Tider beskjeftigede sig med stor Forkjcerlighed, omendstjsnt med ringe Veiledning af Andre . Tegninger af Mennesket udgjorde hendes Indlingsstudium , og Originalstizzer af hendes egen Haand ligge til Grund for de Billeder , der ere foiede til den engelske Udgave af hendes Skrifter . Overhovedet var hun en i enhver Henseende meget selvstcendig Forfatterinde , og har i ingen Henseende med Hensyn til hendes Skrifter at takke Andre for Noget . Ogsaa Vorrecturen besorgede hun selv , og en eneste Gang , da en Anden gjorde det , indsneg sig en Mengde Trykfeil ; dette var Tilfeldet ved de forste Udgaver af to smaa Afhandlinger .

<h4>1329</h4>gode Vrempel for Die , som er den fornemste Fordeel af den huslige Opdragelse , saa gav min <b>Moder</b> endelig efter , efterat min Fader havde lovet at opfylde heudes Vnste om at jeg ikke skulde sendes til en Skole , hvor der var et altfor stort Antal Drenge ; dog forbeholdt han sig Valget . Forstanderen for det Institut , som han valgte , besad alle de Egenstaver , der vare nødvendige , med Undtagelse af en eneste , som min <b>Moder</b> satte hsiest . Han var en Mand af store Talenter , grundig <b>Lcrrdom</b> , og en fim og dannet Smag ; han havde ogsaa vcrret meget iblandt Menneskene , og havde derfor en Werdensmands elegante Manérer . Ha » modtog blot et bestemt Antal af Pensioncrrer , og Nogle af mine Kammerater vare af Landets forste Familier . Under en saadan Larer , der ganske offrede sig for sine Elevers <b>Uddannelse</b> , og forstod at vcrkke min Interesse for de Studier , som han selv satte meest Priis paa , gjorde jeg saa hurtige Fremstridt , at jeg snart erhvervede mig hans Gunst i en hoi Grad . Det laae ogsaa i min Charakter at strcrbe efter at blive elsket af dem , som omgave mig ; derfor erhvervede jeg mig snart ogsaa mine Kammeraters Kjcrrlighed . Kom jeg hjem , saa blev jeg smigret fra alle Kanter ; jeg opdagede , at jeg var min Faders Stolthed og Haab . Min Moders Kjcrrlighcd til mig syntes at have voret under min Fravcrrelse . Hun ssgte dog paa det Omhyggeligste og Vmmeste at overbevise mig om , at ingen <b>Kundskaber</b> , ingen <b>Dannelse</b> , ingen Agtelse , Kjcrrlighed og Beundring , som Andre kunde <b>vise</b> mig , havde nogensomhelst Vcrrd uden Religion . Hun sagde , at Alt dette var kun Ting , hvilke Djcrvelen brugte som Midler til at ncere Stoltheden og Hoffcrrdigheden i det menneskelige Hjerte , og lokke Sjcelen i den evige Fordervelse . Jeg hsrte paa hende , fordi jeg elssede hende , og ikke vilde foraarsage hende nogen Kummer . Min Lcrrer var alligevel en Deist , og jeg havde lcrrt af ham at bctragte alle Neligionssystemer , enhver Religion , undtagen den naturlige , som Prccsteopfindelse og Bedrag, hvoraf kun svage og « vidende Folk lode sig bedaare . Jeg blev i denne Pensionsanstalt indtil jeg drog til Universitetet . Du kan neppe tcrnke dig et Menneske , kjerre Katharine , som kan vcere mere übekjendt med Werden og den mennesteligc Natur ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014031808079'target=_>Besser, W.F., 1862, Johannes's Evangelium</a><h4>1192</h4>Dragen . „ Guds Ord viser mig Vcien , Guds Ord driver mig , " saalcves beskrev engang for migen kjcrr , enfoldig <b>Kvinde</b> sin Gang til Frelseren og lians Samfund . Denne <b>Kvinde</b> blev dragen as Faderen . Dragen — det er et undcrstjsnt Udtryk for den guddommelige Kicrrl.igheds hemmelighedsfulde Arbeide paa de med Brsde betnngedc , i det Jordiske nedsnnwe , fra Himmelen fjernede Mennesker sHsis . 1 , Jer . 31 , 3 : Hos . 11 , 3. 4. ) . Mennesket ligner en under tung Last i Skarnet nedsunken Vogn . og den gnddommclige Kiarligbed er det stoerke Forspand , som drager ham opad og fremad . Igienncm Menneskehedens hele Historie lige til Christi Aabenbarclse , fra Adam lige til Simeon og de <b>Vise</b> fra Ostcrland , gaaer denne Faderens Dragelse til den Ssn , fom skulde komme : og igjcnnem Menneskehedens hele Histostortc siden Christns , fra de Disciple , som Johannes den Dsber henviste til Guds Lam , lige til dci Barn , som den Dag idag dsbes . og lige til den sidste Synder , som stal sukke efter en Forlsser , udstrækker sig Faderens Dragelse til den Ssn , som han har udsendt * ) . Kjcrrlighcden sender , Kjcerligheden drager . Ingen kan komme til Ctnistum uden denne Faderens Dragelse . Ilderne overdsvcde ved sin ren den naadige Kalden , som nu just vcderfores dem derved , ai de fa ae Ssnnen , og derfor var Gnds Kjcerlighcds Dragen forgjoeves paa dem . „ Ligesom Magneten ikke trakter Alt , men knn Jernet til sig , saaledcs maa der i Mennesker vcere en <b>vis</b> bindets Beskaffenhed tilstede , naar Guds Dragen stal have Fremgang " ( Theophnlaky . Hvad er da dette Den , som sordres l Bibel- Gud ? sandede ' orchristelige Indvirkning itte dlot paa Israel , men ogsaa paa Hedningeverdenen , er " en Padagogie sen > lp- Dragelse > t , I l ' l ' nstnm / Den dar sit Hen ' vn til Tidens Fvlde , tonnens Melinestedliven , sit <b>Maal</b> i dette store verdenshistorie ste Tidspu , : kls Hidssrelse . i Menneskehedens Tilberedelse til det . Og just dertil sigter ogsaa bele den > Tiden eftcrs ' slgende g : ! ddom , ne ! ige Indvirkning paa Verden , forsaavidt soin " den havde Frelsens Indforelse i Mennesteoeden , Folkenes og den Enleiles Heinorelse til den , og overvindelse af de Hindrin- - ger , der staae samme imod , til <b>Maal</b> ; Alt er Faderens Dragen til nonnen . " Thmnasius ' s Dogmatik 11.

<h4>2700</h4>troe paa vor kjoere Herre og kjende ham , hvorledes han elsker Sine , som hoenge ved ham , saaledes , at han meget vel kan taalc og finde sig i deres Barnagtighed og Svaghed , Skrobcligheder og Uforstand , for at han ved saadan <b>moderlig</b> Sagtmodighed og Taalmodighed kan beholde dem hos sig og stedse mere styrke dem " ( Luther ) . V . 31. Jesus svarede dem : Nu troe I ! Han veed i Sandhed alle Ting ; han vecd , at Disciplene troe paa ham , at deres Hjerte nu , trostet imod al Skrcek , hamger ved ham ( Cap . 14 , 1 ; sml . 17 , 8. ) , hvorvel han tillige veed , at der paa dette „ Nu " snart stal folge en Forfærdelsens Time , at de gjennem Vedrovelsen mane trcenge igjennem til den fuldkomne Troesglcedes <b>Maal</b> , og at i denne Bcdrovelfe deres endnu tiloversblevne Vantro stal forvolde dem tusinde Smerter . Dog , ligesom Frelserens Forbon for hans kjcere Petrus ( Luk . 22 , 32. ) blev til det reddende Reb for den Faldne , faaledes tjente dette venlige Trssteord : „ Nu troe I ! " alle Disciplene til Lys i den morke Time , hvis Frembrud Herren nu forkynder dem : V . 32. See , den Time kommer og er allerede kommen , at I skulle adspredes , hver til Sit , og forlade mig alene ; dog jeg er ikke alene , thi Faderen er med mig . Nn troe I , men see , den Time kommer , at Eders Tro , som har samlet Eder om mig . vil <b>vise</b> sig at voere for svag til at holde Eder samlede ; I stulle adspredes, hver til Sit , til det , I for min Skyld have forladt . Med Ordet : adspredes peger Herren tilbage paa Ordene i Sachar . 13 , 7. om den slagne Hyrde og den sig adspredende Hjord , som han Matth . 26 , 31. udtrykkelig anforer . Vistnok var det Disciplene tilladt i denne Vtorkets Time at flye , ja udtrykkelig af Herren felv udvirket dem ( Cap . 18 , 8. ) , thi hverken kunde de nu taale at lide Martyrdoden ( Cap . 13 , 36 : 18 , 9. ) , ei heller var det deres eller noget Menneskes Sag at dele Forloserens Forsoningslidelser ; og dog var den Bedrsvelse , de solte , da de maatte grcede , adspredte hver til Sit , blandet med Smerte over , at de paa en syndig Maadc havde forladt deres elflede Herre , thi ikke deres

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008092603012'target=_>Møller, Poul, 1842, Efterladte Skrifter</a><h4>2086</h4>De senere Kynikere vare raae Betlere , som savnede al vidensiabelig <b>Dannelse</b> , og for deres uforskammede Opforsel bleve kaldte Hunde . Hvorvidt de dreve deres Frcekhed og Trods imod al Anstcendighed, kan sees deraf , at Krates fra Theben og hans <b>Hustru</b> , der ogsaa var Philosoph og Kynikermde , parrede sig offentlig paa Torvet . De vilde dermed <b>vise</b> , at den <b>Vise</b> ikke skulde stamme sig ved offentlig at foretage Noget , som kunde bestaae med Dpd .

<h4>2701</h4>Oekonomiken handler om Huusvcesnet eller det selstabelige Forhold mellem Mand og <b>Kone</b> og Bsrn og deres Besiddelser . <b>Formaalet</b> for Familielivet er at realisere det Gode saaledes , som det kan realiseres i Familieforhold . Hvad der ncermest forer Mand og <b>Qvinde</b> sammen , er Attraa efter at vedligeholde Arten ( Drift til Udodelighed , see S . 483 ) . Men deres Samlivs Bestemmelse er ikke blot denne ; de ere bestemte til at <b>vise</b> hinannen Velvillie og yde hinanden gjensidig Hjelp . Forholdet er ikke blot et naturligt , men

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111500040'target=_>, 1851, Forordninger, aabne Breve, Placater m. m. for kongeriget Norge i Tidsrummet fra 1648-1813</a><h4>1063</h4>min Mund , thi du Herre gjorde det , Ps . 39. v. 10. Her kommer sommetid en af Moders-Liv , som mangler noget paa sine Lemmer , som den Blinde . Joh . 9. v. 1. , den Lamme , 3. v. 2 , , sommctider en , som har for mange Lemmer , som den lange Mand , der havde ser Fingre paa hver Haand og ser Tcrer paa hver Fod , 2 Tam . 21. v. 20. sommctider kommer og et dedt Menneske frem , som har intet Liv af Moders Liv ; men hvo ville derfor tratte med sin Skabere og sige til Herren- hvad gjorde du ? thi Vce den , som siger til en Fader : Hvad avlede du ? og til en <b>Moder</b> : Hvad fedde du ? Gs . 45. v. 9. w . Tanker I med <b>Qvinden</b> i Sarepta . at Gud er Eder særdeles vred derfor , og at dette er stret Oder , at Eders Misgjerning skulle ihukommes , og Eders Ssn ( Haattcr ) omkomme . 1 Reg . 17 v. 18. saa at I tykkes I har godt Skjel til at sige med Rebekka : Har Gud velsignet mig , hvorfor er da dette ? 6 en . 25 v. 22. : da vider , at Guds Hom stal altid begynde af Guds Huus , efter Petri <b>Lardom</b> , 1 Pet . 4. v. 17 , og de bedstc Born revser Gud ofte allerhaardest . Hebr . 12. v. 6. Seer paa Guds egen Ssn , som var ligcre en Orm end et Menneske , og havde intet brudt dertil , Ps . 22. v , 7. Seer paa de Hellige , hvad de har udstaaet . seer paa lob , lob . 1 v. 18. Seer paa Paulum , den udvalgtc Guds Apostel , 2 Cor . 12. v. 7. Ja seer paa alle Guds Bern , stulle de ikke alle lide med hannem . ( Christo ) , om de ellers ville arefuldgjsres med hannem ? Rom . 8. v. 17. Lagger Christi Ord vet paa Hjerte , som han sagde til Pharisceerne , der svurgte om det Menneske , som var blind fsd af Moders Liv . at hverken han , eller hans Foraldre havde syndet dertil , men at Guds Gjerning stulle blive aabenbaret paa hannem , Joh . 9. v. 3. , og troster Eder ved Pauli Ord , at alting stal tjene Eder til Gode , om I ellers frygter og elsker Gud , Rom . 8. v. 28. Lad vom Eders Barn var svagt i Fsdselen . det er dog saligt ved Igjenfedelsen , og skal arefuldgjsres ved Opstandelsen . hvor ingen Rynke eller Smitte stal sees mere . Eph . 5. v. 27. Thi det , som saaes i Forkrcrnkelighcd , stal opstaae i Uforkrcrnkelighed . Kraft og Herlighed . 1 Cor . 15. v. 42. 43. Er end Eders Barn borte for Eder , saa er det dog hos sin og alles vores bedste Fader i Himmelen ; savner I det her paa Eders Sljod , glader Eder , at det saa vel hviler der i Abrahams Skjsd , Luc . 16. v. 23. Er det ikke mere her iblandt Menneskene, da er det der iblandt de Udvalgte og Englenes mange Tusinde , Hebr . 12. v. 22. Er det end her dsdt for os , lever det dog der i evig Fryd og Glade , hvor intet ondt mere stal rsre det , 8 » p. 3 v. 1. Nu har I ikke fornsden at sorge videre derfor . Jesus har taget Eders Barn til sig , at det stal vare hvor han er og nyde hans Salighed evindelig . Saa vcrrer nu tilfreds med Guds Villie og gaaer ind i Guds Huus med Lov og Tak , styrker Eders Sjal i Gud og priser hans Navn . som gjor alting saare vel . Gjor I det . saa stal hans Velsignelse hvile over Eder . og I stal ste det med Glcrde alle Eders Lives Hage , hvilket jeg saa troligen msker over Eder , som jeg alvorligen i Tro og Tillid beder for Eder : Fader vor , du som er i Himmelen : c . Gud styrke Eder med sin <b>Aand</b> , og lade Eder hver Hag see og smage . hvor sod Herren monne vare !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111903009'target=_>Hansen, Maurits, 1858, Mauritz Hansens Noveller og Fortællinger</a><h4>100</h4>IndpaS , maatte man lidet have iagttagct Hedebrandts Karakteer , Utringer og hele Maade at vwre paa . Om den Mand maatte man uden Tvil antage , at kun en eneste Lidenssab laa sammenfoldet i hans fortsrrede Inderste , som kun under i de Tider utrolige Omstcendigheder kunde bringes til Virksomhet » ; og denne Lidenssab , der i enkelte Gnister sijod frem af den kolde Sten-Figur , hvis Krystall ! sati on dog selv antydede Principet for dens <b>Dannelse</b> , var Wrgjerrighed . Madame Hnssing havde ganske for sig selv en Mening , som hun kun halvt lod Bolten moerke : at nemlig han var den uskyldige Aarsag i Henriettes Forbitrelse imod Kathrine . Vist var det , at Kathrine altid med levende Interesse omtalte deres Barndomsven . Men det usandsynlige i denne Grund til laloust blev da rigtignok indlysende , naar man erindrede sig , hvilken underordnet Rolle i Samfundet det unge , übetydelige Menneske syntes at maatte komme til at spille , og at intet saa lidenskabeligt Forhold endnu ved Andreas ' s Ophold kunde have sundet Sted , hvorover den fremblomstrende , rige og smukke Froken skulde kunne onske engang i Tiden at kapre en saadan Elsker fra den stakkels Hnsmandsdatter , for hvem han maasie netop kunde blive et passende Parti . — De her anforte Visninger maa staa der for det hele Forraad af lige saa usandsynlige , med hvilke Tanten i sin uendelige Vidloftighed plagede Bolten . At Kathrines Vedstemoder , den gamle Enke , Malene , maatte have en vcrseiitlig Del i det huslige Misforhold , som efterhaandcn ovstod ved Oberstlejtnantens Godhed for den forcrldrelose <b>Pige</b> og Henriettes Uvilje imod hende , lod sig vel stutte , men ingenlunde forklare , af den Omstcrndighed , at den gamle Malene var bleven et Indlcrg i Huset , efter at have fraflyttet sin Plads , just ved den Tid , da Forholdet imellem <b>Pigerne</b> blev spcrndt . Henriette syntes nu at hade , nu at elske den gamle , stille <b>Kone</b> , som ved sit underlige , sagtfcerdige , hemmelighedsfulde Vcrseu gav alle Mennesker nok at tale om . Oberstlojtnantcn havde ingen Godhed for den svagelige Gamle , men han havde stedse baaret en <b>vis</b> Omhu for hende , og han spurgte ofte om hende , gav hende regelmcrssig til Hsjtiderne en Gave og viste hende , efter at han havde anvist hende Bolig og Ophold i Huset , en Omhu , som ikke stemte overens med den Maade , hvorpaa Manden ellers behandlede sit Tyende .

<h4>2346</h4>Ingen siulde lettelig tro , at den blege , magre , mgrke Kirsti Haitler , med det matte Blik og den usikre Holdning , i sin Tid havde vceret en udmcrrket Skjsnhed . At hun havde tabt fin Stemme , erfarede Lceseren tilfcrldigvis . Men med Skjsnheden og Stemmens Velklang havde hun ikke tabt den heje , crdle Tcrnkemaade , der bar hende over det stormende Livshav , med Blikket vendt lmod en bedre Verden . Hun havde i fin fprste Fremblomstren vundet den unge Kontorbetjent Barths Kjcerlighed og elskede ham igjen . Forholdene hindrede ham i for det fyrste at vedkjende sig sin Wede , og hun fandt sig med Taalmodighed deri og delte med ham den hemmelige Kjcerligheds stjaalne Glceder . Men da han ef , ter nogle Aars Ophold i Udlandet var kommen tilbage , og i en agtet Stilling blev msdt fra alle Kanter med Udmcrrkelse , trak Kirsti Povelsdatter sig tilbage fra ham . Den Alvorlighcd , hvormed han paastod , at deres Forbindelse stulde erklcrres for Verden , manglede , — det fslte hun vel , hin Kjccrlighedens inderlige Fylde , hvormed han som Yngling havde tilsvoret hende , at hun var hans Hjertets alt. Hun erkjendte , at kun den retssafne Mands Pligtfglelse nu bestemte ham til at forbinde sig med en <b>Pige</b> , der var langt under hans Stand og <b>Dannelse</b> . I nogen Tid kccmpede hun en svcrrKamp med sig selv : men jo mere hun iagttog Barths Forhold til den hende fremmede store Verden , hvori han den Gang med Livsglcrde deltog , desto mere blev hun overtydet om , at hun ikke kunde trcrde op ved hans Side , uden at nede ham til at forsage sine Liges Omgang . Hans svage Modgrunde , hans Forsikring , at hun alild vilde agtes og vurderes af dem , hvis Mening var ham vigtig , og at han kun lidt skjottede om den svrige Verdens Dom , — bestyrtede Pigen i hendes Beslutning . Havde han med Mod og Glcrde forsikret hende , at han vilde gjore hende gjeldende i den Kreds , hvortil han hsrte , — at hendes <b>Aand</b> og Hjerte snart vilde hcevde hende Pladsen , at hun let maatte kunne tilegne sig Kongsbergerindernes den Gang temmelig tarvelige Verdcnsdannelse : — da havde hun maaffe ikke med et sønderknust Hjerte frasagt sig sine ssde Ungdomsdromme og sine uomtvistelige Rettigheder.

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016060348148'target=_>, 1862, Vor Herres og Frelsers Jesu Christi nye Testamente</a><h4>1832</h4>22. Der de sagde sig at være <b>Vise</b> , bleve de Daarer , I C0r.1 , 20. 33. og have forvendt den uforkrænkelige Guds Herlighed til Lighed med Billedet afet forkrænkeligt Menneske og af Fugle og affirefoddede Dyr og af Orme . 34. Derfor haver og Gud givet dem hen i deres Hjerters Begjæringer til Ureenhed , saa at ( de ) vancere deres egne Legemer indbyrdes . Ap.G .14 , 16. l 25.DehaveforvendtGudsSandhed til Logn og have dyrket og tjent Skabningen over Skaberen , som er velsignet i Evighed , Amen ! 36. For denne Sags Skyld haver Gud givet dem hen tll skjamdige Lyster; thi baade deres <b>Qvinder</b> forvendte den naturlige Brug til den unaturlige , 37. og desligeste bleve ogsaaMcendene,som forlove Qvindens naturlige Brug , optcrndte i deres Lyst til hverandre , saa atMcend med Mænd ovede Uteerlighed og fik deres Vildfarelses Lon , som det burde sig , paa dem selv . I C0r.6 , 9. 38. Og ligesom de ikke holdt ( det ) for godt at have Guds <b>Kundskab</b> , saa gav Gud dem hen til et Sind , som intet duer , saa at de gjore det Usommelige , 39. opfyldte med al Uretfærdighed, Horeri , Skalkhed , Gjerrighed , Ondstab , fulde afAvind , Blodtorst , Trcette , Svig , Vanart , 30. ^ retudere , Bagvastere , Guds-

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011012406002'target=_>Scherr, Johannes, 1876, Almindelig Literaturhistorie</a><h4>679</h4>er Produktet af en Tid , da Dannelsen og det aandelige Liv var gaaet over fra Fyrsternes og Adelens Borge til de frisk opblomstrende Stæder , og da Borgerstanden var traadt i Stedet for det udartede Ridderskab som Kulturens og følgelig ogsaa dens nationalliterære Udtryks Bærer . Efter den ridderlige Fantastik fulgte med det 15 de Aarhundrede den borgerlige Forstandighed . Uen Maade , hvorpaa Sangkunsten dreves i Mestersangerskoleme , havde meget prosaisk Haandværksmæssigt ved sig . og Mestersangens Kunstværd er i det Hele taget meget ringe , men denne hæderhge Borgerligbed har dog i sin Kreds lagt og plejet mangen en Spire til <b>Dannelse</b> , og man kan ikke fråkjende dens ofte højst forunderlige Udgydelser en <b>vis</b> andægtig troskyldig Hengivelse i Æmnet , en <b>vis</b> Varme i Følelsen . Forbilleder for Mestersangen bleve de senere Minnesangere ; dermed er Indholdet allerede givet : det var en lyrisk udpyntet , væsentlig religiøs Gnomepoesi , der tabte sig i den skolastiske Dogmatiks bundløse Sand og senere tog den lutherske Orthodoxi til Rettesnor . Det første Lav af borgerlige Sangere skal Frauenlob have stiftet i Mainz ; en gammel Tradition fører endogsaa Mestersangens Oprindelse tilbage til Otto Ts Tid , til Aar 962. Den ældste bekjendte  Tabulatur " , d . v. s . Lovbog , indeholdende Regler for Prosodi , Metrik og Rhetorik , er Mestersangerskolens i Strassburg fra 1493. Efter at Karl IV havde skj senket Mestersangerne Korporationsrettigheder og Fribreve ( 1378 ) , steg Antallet af Sangerskoler rundt om i Byerne betydelig , fra Sydtydskland bredte de sig mod Øst til Breslau , mod Nord til Danzig . I én By slog Mestrene i et enkelt Haandværk , i en anden de sangkyndige Mestre i forskjellige Haandværk sig sammen til et Sangerlav , der Søndag Eftermiddag  sang Skole " paa Raadhuset eller i Kirken med det hæderlige Borgerskab , Mænd og <b>Kvinder</b> , som Tilhørere . Livligst trivedes Mestersangen i det 16 de Aarhundrede ; den overlevede Trediveaarskrigens Storme og holdt sig til langt ind i det 18 de Aarhundrede . Den sidste Sangskole holdtes i Nurnberg 1770 , men i Ulm overgav de sidste Epigoner af Mestersangen først deres Tabulatur og Sangbøger til den derværende Sangforening i 1839. Den frugtbareste saa vel som den betydeligste af alle Mestersangerne er Hanns Sachs , den fortræffelige Niirnberger Skomager , som vi siden ville komme til at omtale nærmere .

<h4>1183</h4>Henimod Slutningen af det 18 de Aarhundrede opstod der i Holland en ny Digterslægt , hvis Medlemmer dog kun sjelden forlod Literaturens gamle , vel tilgaaede Spor . Vel lærte man den engelske og tydske Literaturs Skatte at kjende , og navnlig begyndte den tydske Lyrik at indvirke paa den hollandske , men i det Hele taget beholdt dog den franske Parykstil Overvægten . Paa Skuepladsen herskede den opstyltede Pseudoklassik , som navnlig Sybrand Fei tam as ( død 1758 ) stive Tragedier <b>vise</b> , og de reformistiske Bestræbelser , der udfoldedes af N . S . Winter s og hans ovenfor nævnts <b>Hustru</b> samt af den lunefulde Komedieskriver Pieter Langendijk ( død 1756 ) , formaaede ikke at trænge igjennem . Der var vel Digtere , der havde den bedste Vilje til at forynge deres Lands Literatur ved at gjenoptage den gammelnederlandske Nationalstil ; der var Jakob Bellamy ( 1757 — 86 ) og Rhynvis Feith ( født 1753 ) , der med Goethes Werther som Forbillede forsøgte sig i den sentimentale Roman og viede Nationalhelten de Ruyter en begejstret Sang , men de havde ikke Talent og Kraft nok til at naa <b>Maalet</b> , og Digterinderne Elizabeth Wol f f ( 1738 — 1804 ) og Agathe Deken ( 1741 — 1804 ) skrev vel nogle

<h4>2075</h4>I Centrum af den nationale Aandsudvikling i dotte Tidsrum staar Georg Stjernhjelm , Grundlæggeren af den svenske Digtekunst . Han var Søn af Bjergmanden Olof Marqvardsson i Dalarne og blev født 1598. Han lagde tidlig baade Flid og <b>Begavelse</b> for Dagen , og efter at han havde fuldendt sin <b>Uddannelse</b> under et længere Ophold i Udlandet , gjorde Gustav Adolf ham til Lektor ved Gymnasiet i Vesteraas , hvor han havde faaet sin første Opdragelse . Det vårede dog ikke længe , før hans omfattende <b>Kundskaber</b> fandt en større Virkekreds i forskjellige tildels praktiske Stillinger . Gustav Adolf ophøjede ham i Adelsstanden , og han aflagde ved denne Lejlighed Navnet ( Goran ) Lilje , som han havde baaret efter sin Farfaders <b>Moder</b> , der var af adelig Byrd , og antog det , under hvilket han er kjendt i sit Lands Literaturhistorie . En Tid var han Dronning Kristinas Hofdigter og stod i høj Gunst hos hende , men da der opstod grundet Mistanke om , at han tilhørte Oppositionen , faldt han i Unaade . Sin Anseelse beholdt han til det sidste , men han døde som en fattig Mand 1672. Stjernhjelm indtager en lignende Plads i den svenske Digtekunsts Historie som Martin Opitz i Tydskland , idet han var den første , der gjorde de nye med Renaissancen opstaaede Principer gjældende og lagde Grunden til en virkelig Kunstdigtning , idet han med <b>Bevidsthed</b> om Betydningen af en national Poesis Udvikling paa dette Grundlag søgte at tillæmpe disse Principer

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009031612001'target=_>Lous, M.C. Theodor, 1865, Haandbog i den christelige Børnelærdom for Confirmander og Confirmerede</a><h4>243</h4>„ Kom Sabbatens Dag ihu at helligholde den ; sex Dage stal du gjere al din Gjerning ; men den syvende Dag er Sabbat for Herren din Gud , da stal du ingen Gjerning gjere , hverken du , eller din Son , eller din Datter , din Svend eller din <b>Pige</b> , dit Dyr eller den Fremmede , som er i dine Porte ; thi i sex Dage gjorde Herren Himmelen og Jorden , Havet og alt det , som er i dem , og hvilede paa den syvende Dag , derfor velsignede Gud Sabbaten og helligede den " . Den ugentlige Helligdag er altfaa indfat af Gud felv , gruudet paa en guddommelig og ikke blot pna nogen menneskelig Anordning ; og maa vi i disfe Ord , hvormed Gud felv anordner denne Ugefcst , legge neie merke til de Ord : „ Gud velsignede og helligede den " . At Gud velsignede den syvende Dag vil sige , han gjorde den til en velsignet Dag , en Dag , paa hvilken han ifer vil velsigne og lykkeliggjere Menneskene , idet han ifer paa den Dag vil aabnc os sine himmelske Forraadskamrc og lade sine uåndelige Velgjerninger i rige ^ tigt <b>Maal</b> tilflyde hver den , som selv vil soge og modtage dem , hvilket « armere antydes ved det andet Tillag , han „ helligede den " . Gud helligede den syvende Dag , man kulder alle de jordiske Ting , hvis Bestemmelse er gudeligt Brug , for hellige Ting ; i denne Betydning helligede Gud den syvende Dag ; idet han bestemte den ti ! gudeligt Brug , til Gudstjeneste, til haus Navns Forherligelse , hans Riges Udbredelsc paa Jord . Paa den Dag skulde altsaa Menneskene i Eardeleshed tjene Gud , og derved adstilles denne Dag fra dc ourige Ugedage , paa hvilke man stal gjere sin Gjerning , altsaa ogsaa tjene sit jordiske Behov Ester dette stal altsaa denne Dag vere en Hviledag for Legemet ; men en Arbeidsdag for <b>Sjelen</b> til dens Helligelfe , og faaledes bliver der den nsieste Forbindelse mellem Hviledag og Helligdag. Isderne helligholdt den syvende Dag i Ugen eller vor Leverdag , fordi Gud paa den Dag fuldendte sit Skaberverk og , som del heder i 1. Moseb . 2 , 3. velsignede den syvende Dag og helligede den , fordi han hvilede paa den fra al sin Gjerning , som han havde fiabt og gjort ; fordi denne Dag senere hen , ved Lovgivningen paa Sinai , med tydelige Ord blev givet lederne af Gud , at den skulde vere deres Sabbat , efr . 2 Mofeb . 20 , 8 — 11 ; og endelig fordi da Folket i Mofe sidfte Dage atter fornyede deres Pagt med Herren ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092203003'target=_>, 1859, Almindelig Norsk Lovsamling</a><h4>3628</h4>de nu selv stal igjentage den Pagt , som Fadderne ved Daaben have gjort for dem , og at de nu vel maa prove sig for Herren , hvorledes de have holdt deres Daabes Pagt , og om de befinde sig i den Stant » , , som Gud ved Daaben har sat drm udi , saasom de ellers vare uverdige til at lomme frem for Herren og den kristne Menighed . 9. Lereren stal 8 Dage tilforn give Menigheden den forestaaende Konfirmation tilkjende ; men paa Konfirmationsdagen stal han i sin Prediken tale om denne Sags Vigtighed , og opvekke Menigheden til hjertelig Forben for Konfirmanterne , og selv paa Predikestolen paakcilde Gud tilligemed Menigheden len bevegelig Bon . 4 U . Efter Prediken og strår fer Konfirmcttionen synges : , ' , I Jesu Navn " ic . eller : „ Kom Gud Skader " le . Derpaa gaa Vernene op for Alteret , eller Lereren gaar ned paa Kirkegulvet , hvor Drengene staa paa den ene og <b>Pigerne</b> paa den anden Side , og da begynder Forretningen med en Ven , holder en kort Tale , saasom hans Embede da fornemmelig er , ved Sporgsmaal at give Vernene Anledning til at gjore Menigheden Nede for det Haab , som er udi dem ; men i Ouerhoringen selv gaar han enten Kcttekismi 5 Hovedparter igjenuem , eller tager Ealiggjoreisens Orden for sig eller og Ponitentsesvarterne , og seger at faa dem de evrige Troens Artikler indprentede . Dervaa gaar han fra Barn til Barn , og ikke af Passion eller Affekter gaar et eneste forbi , saasom hvert Varn maa lade Menigheden here , hvad det har lert . Kan et Varn ikke svare til et eller andet Sporgsmaal , gaar han det forbi , og sperger det andet derom , som staar nest ved , og det uden at beskjemme det andet Barn med noget Ord for Menigheden , saasom Lereren overalt herved maa entholde sig fra al Partisthed og Persons Anseelse ; hvis Vernene ikke svare forstaaeligt nok , opmuntrer han dem med Sagtmodighed til at tale heit , og saa Folk kan forståa det . 44. Naar Eramen er forbi , gaar Lereren til Konfirmation i sig selv ; til hvilken Ende han foreholder Vernene deres , Daabes Pagts Vigtighed . og opvekker dem til vel at betenke , at de nu selv stal igjentage deres Fadderes Leste . Derpaa sverger han enhver iser : Om han af ganske Hjerte forsager Djevelen og alt hans Vesen og alle hans Gjerninger ( hvilket ferst vel forklares , hvad det betyder ) ; dernest : Om han af ganske Hjerte tror paa Gud Fader , Son og Helligacmd ( efterat han forst vel har forklaret den saliggjorende Troes Natur og Egenstab ) ; for det tredie : Om hau vil blive i saadan sin Daabes Pagt til sin sidste salige Ende ; saa maa han det med et tydeligt Ja bekrefte , og give ham sin Hacmd derpaa ; hvorpaa Lereren legger efter Chrlsti Ercmpel Haanden paa Barnets Hoved , og gjer omtrent saadan Ben : Den treenige Gud , som fordum har cmtaget dig til sit Bci ? n i den hellige Daab , og gjort dig til en Arving til det evige Liv , han opholoe dig i saadan din Daabes Naade indtil din salige Ende ; han give dig Bestandighet » i din Tro , din <b>Sjel</b> til Salighed " . Nåar saadan Indvielse er steet ved hvert Barn , holder Lereren en kort Tale , hvorved Vernene : Imo vises , at dette deres Lefte er en formelle Ed for Herren , hvorfor de saa meget mere ere forpligtede til desto troligere at holde den fornyede Pagt , og aldrig forlade den sande evangeliske Religion , eller i deres Levnet uige af fra dens Sandheder , hvilke de nu have bekjendt og svoret sig til . 2 ciu . Opuetkcs til hjertelig Taknemmelighed mod Gud for den Naade , Gud har skjenket dem i Daaben , og for den dem nu forlenebe <b>Kundskab</b> om den rette Vei til Livet , og at de nu maa vederfares den Naade , at annammes til Jesu Vord . 3 tio . Formanes vel at betenke deres Daabes Pagt , og ved en levende Tro og retstaffen Kristendom lade al Verden se og kjende , at de ere blevne Guds Bundsforvandte . Hvad Tid Vernene derpaa kan gaa til Guds Vord , overlades dels til deres egen Attråa , dels tll deres Foreldres Villie , dels ogsaa til Lererens Godtbesindende . og skjent Lereren maa raade Vernene fremdeles at komme i Katckiscition med andre . Vern , forundes dem dog derudi deres Frihet » . Men dertil stal Lereren hjertelig formane dem , at de aldrig uden sand Prevelse og Forberedelse komme til Herrens Bord , og for alting strebe i Omvendelse og Ben at bringe det dertil , at de den ferste Gang maa gaa dertil med inderlig Hjertens Bevegelse , saasom Saadant vilde give et kraftigt Indtryf i deres Sjele , saa lenge de , leve . Er der flere end een Lerer i Menigheden , blive Vernene ved Konsirmationen givet Frihet » at gaa til Skrifte hos hvilken Lerer de vil . Naar denne Forretning med Vernene er til Ende , stal Lereren opnmntre Menigheden til hjertelig Tatnemmelighed for den store Naade , disse Vern er vederfare ! , samt til Forben for Vernene , at de maa vore og blive bestandige i alt Godt ; fremdeles opvekker Lereren samtlige Tilhereres Samvittighet » , hvorvidt de have holdt deres Daabes Lefte , opmuntrer dem til at omvende sig , og efter Jesu Villie at blive som Vern ; Foreldrene formaner han at takke Gud for deres Bern , at de stal styrke dem i alt Godt , holde dem tll at tomme i den offentlige Kateliscition . og altid foregan dem med et godt Erempel , og ei med forargelige Ord og Gjerninger drive den Helligacmd af deres Hjerter . Hvorpaa han leser Kunfinnationsbennen samt . Fadervor og Herrens Velsignelse ; siden synges : „ Mit Barn frygt d.en " : c . eller „ Gud Fader udi

<h4>6740</h4>bestemte , ssulle ikke under nogetsomhelst Paassud eller Forevendlng kunne anholdes . Negre , som i Egenssab af Betjente eller Sefolk befinde sig ombord paa bemeldte Farteier , kunne i intet Tilfelde ansees som gyldig Anledning til Anholdelse . 2. Saa ofte et til ovennevnte Oiemed beordret Kongeligt Krigsssib <b>moder</b> et Handelsssib , som er Visitation underkastet , ssal denne sse paa den mest hoflige Maade , c > g med lagttagelse af al den Opmerksomhed , som tyende uenssabelige og allierede Nationer ere hinanden skyldige . I intet Fald kan en saadan Visitation foretages af en Officier af mindre Grad , end en Leitnant ved Marinen . 3. De Kongelige Krigsssibe , som dertil ere beordrede , og som komme til al anholde et Handelsssib , i Overensstemmelse med Indholden af nerverende Instruktion , siulle lade Ladningen hel og urort forblive ombord , ligeledes Skipperen og i det mindste en Del af Fartoiets Mandssab . Den Officier , som iverksetter en saadan Anholdelse , sial siriftligen afgive en Forklaring paa Embeds Vegne , hvori omhandles den Tilstand , i hvilken han har truffet det anholdte Skib , og de Forandringer , som dermed kunde have tildraget sig . Han sial til Skipperen paa det anholdte Fartoi afievere en undersircven Fortegnelse paa de ombord i Skibet fundne Papirer , saavelsom paa Antallet af de Slaver , suiit sammesteds forefandtes i bet Oieblik , Anholdelsen foregik . Negrene ssulle ikke udssibes , forend de Fartoier , paa hvilke de ere ombord , ere ankomne til det Sted , hvor Anholdelsens Lovlighet » ssal paadommes . Hvis alligevel gyldige Grunde , hentede fra Neisens Langvarighed , Negrenes Sundheds-Tilstand eller andre Aarsager furdre , at de blive udssibede enten alle , eller for en Del , kan Kommandolen for det Krigsssib , som har ivetfsnt Anholdelsen , paa sit Ansvar foretage en saadan Udssibnlng, dog at Nodvendigheden deraf bevidnes ved et i lovlig Form udferdiget Dokument . Vil ag Litr . v. Reglement for de blandede Domstole t . De blandede Domstole , som efter Traktaten af Dags Dato ssulle oprettes , have tll <b>Formaal</b> at bedomme Lovligheden af Skibes Opbringelse , der ifolge bemeldte Traktat anholdes af begge Nationers Krydsere . Bemeldte Domstole siulle domme ved endelig og inappcllabel Dom , i Overensstemmelse med Traktatens Forskrifter . Rettergangen bor vere saa summarisk sum muligt , og Domstolene bor ( saavidt de sinde saadant gjerligt) afsige Dom inden 20 Dage , fra den Dag at regne , at det anholdte Skib indkommer i , de Havne , hvor Domstolene befinde sig . De siulle for det ferste bedemme Anholdelsens Lovlighed , og for det andet ( hvis det opbragte Fartei maatte vurde frigivet ) den Erstatning , sunt bor tilkjendes det upbragte Fartoi . Og bestemmes det herved , at Afsigelse» af den endelige Dom under enhver Omstendighed ikte fan opsettes , paa Grund af Vidners Fraverelse eller Mangel a f andre Veviser , over en Tid af tyende Maaneder, med mindre det sier paa en af de angjeldende Parters Forlangende , i hvilket Fald og under Betingelse af , at Parterne stille tilstrekkelig Sikkerhed for , at de selv paatage sig alle Omkostninger ug Felger af Udsettelsen , Domstolene have Magt tll efter Forgodtbefindende at tilstaa en yderligere Udsettelse , sum itke maa uuerstige fire Maaneder . 2. Enhver af disse Domstole ber vere sammenfat paa felgende Maade : Begge de hoie kontraherende Parter ssulle hver udnevne en Dummer og en Voldgiftsmand , hvilke ssulle vere bemyndigede til at gjore sig bekjendte med og uden Appel paadomme alle de Sager, som opstaa af Farleiers Opbringelse , hvilke ifolge Bestemme ! serne i Traktaten af Dags Dato undergives deres Kjendelse . Alle vesentlige Dele af de Netshandlinger . som foregaa ved disse blandede Domstole , ssulle nedskrives enten paa Svensk eller Norss , eller paa Engelsk . Dommerne og Voldgiftsmendene ssulle astegge Ed paa , at de ville domme med Nedelighed ug Upartisshed . at de ikke ville tilstaa nugen Furtrinnet , hverken Klagerne eller Opbringerne , ug at de i alle deres Dommerhandlinger uille ^ rette sig efter de i Traktaten af Dags Dato indehuldte Forskrifter . Ved huer Domstol ssal , nåar den sammentrcrder , udnevnes en Sekreter eller Notarius , som ssal hulde et Register over alle dens Forhandlinder , og som , forend han tiltreder sin Post , ssal astegge Ed for Retten paa , at han vil <b>vise</b> AZtbedighed mud samme , ug med Nedelighed behandle alt , hvad der vedkommer hans Tjeneste . I det ved Traktatens 4 de Artilel forudseede Tilfcrlde , ssulle de bestandige eller tilfeldige Lenninger , som tilstaaes de blandede Dumstules Medlemmer, udbetales af deres respettive Regjeringer . Sefreterens eller Notariens s ! en ved den i Vestindien oprettede Domstol ssal betales af Hans Majestet Kongen af Sverige og Norge , ug Sekreteren ved den paa den Afrikanske Kyst uprettende Domstul vil blive at lonne af Hans Vrittisse Majestet . De tilfeldige Omtustninger ved bemeldte Domstole blive at bestride med det Halve af hver Negjering . De Omkostninger , som efter Regning godtgjeres den Embedsmand , som har med Modtagelsen ug Besergelsen af de anholdte Skibe saavelsom med Iverksettelsen af Dummene at bestille lMir ^ bliil ul tbe Oourt ) ligesum enhver anden Udgift , som foranlediges ved et Farteis Opbringelse , eller

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010111006094'target=_>Arndt, Friedrich, 1876, Kristelige Betragtninger til hver Dag i Aaret</a><h4>151</h4>opfyldte Loven , og ved sin Naadekraft fornyer han nu i alle sine Troende Lysten og Driften til at holde Guds Bud . Deres Lydighed vedbliver vel at vcere mangelfuld , men Hans Retfcerdighed og Lydighed gjcelder for dem hos Gud . Og en Gang kommer den Tid , da de atter stulle oplMes til deres fsrste salige Stand , og blive Ham lige , den Førstefødte af mange Bwdre og se Ham , som han er . — Et Menneske maatte Forløseren altsaa vcere , et Menneske som vi . Det havde vceret let for den almcegtige Skaberhaand , at danne den anden Adams Legeme ligesom den førstes af Jord . Han vilde da vel have vceret et fuldkomment Menneske ; men han vilde ikke have vceret ncermere beslcegtet med os Mennesker , end med Englene . Derfor sendte Gud sinSsn , fpdt af en <b>Kvinde</b> og Loven underdanig . Nu er han i fuldeste Betydning vor Blodsfrcende , vor sanoe Forloser . - - Dog , hvis han , ligesom vi , var bleven fM af en fyndig Fader , saa kunde han ikke have vceret ren og hellig og adskilt fra Syndere . Derfor beredte Gud selv umiddelbart hans Legeme ; derfor beljende vi ogsaa gladelig Troen vaa Ham som den , der er undfangen af den Hellig <b>Aand</b> . Nu , da han var übesmittet af Syndens Gift havde han ikke npdig som Ivversteprcest fprst at ofre for sine egne Synder . Hans Lydighed var fuldkommen , hans Natur uden Plet og Mangel , og hans Offer fyldestgjorde aldeles Loven om Guds Retfcerdighed . — Lader os her falde ned og med cerefrygtsfnld Tavsheo beundre den guddommelige Kjcerligheds <b>Visdom</b> og Magt , med cerefrygtsfuld Tavshed beundre den nye store Skabning vaa Jorden , og dens stille , hemmelighedsfulde og dog saa vel beregnede og hensigtssvarende Begyndelse !

<h4>1106</h4>De ti Dage mellem Kristi Himmelfarts-Dag og Pinse kaldes fra crldgammel Tid i Kirken Ventedagene , fordi det var i disse , at Dissiplene i Jerusalem ventede paa den Hellig <b>Aand</b> . Ogfaa os bonhorer den kjcere Gud ikke altid strax naar vi bede til ham ; han lader os stundom vente lcenge , " som Jesus den kananceiste <b>Kvinde</b> . Han bcerer sig ad ligejom hin Fader , hvis Barn om Foraaret gaar omkring i Haven og ssr Kirsebcer ; Barnet ved , at det er en velsmagende Frugt , og beder derom .  Nej , mit Barn , " siger Faderen ,  de ere endnu ikke modne ; bi , til den rette Tid kommer ! " Naar Hans Time er kommen , hjcelper Herren og hprer sine Boms Naab hverken tidligere eller sildigere . Men Tid og Stund har han forbeholdt sin egen Magt og <b>Visdom</b> . Han forhaler ikke Forjettelsen , om vi end anse det for en Forhaling . Han forhaler den ikke , om det end varer til Natten . Hvad vi bede om som Born i barnlig Enfoldighet » , bliver ofte Wnhyrt i en senere Alder , naar vi nceppe mere tcenke derpaa . Og om end Herren , menneskelig talt , blev os det ganske skyldig her , saa vil hin Dag verre lang nok til at fremstille Lcengden , Vreden , HHden og Dybden af Hans Kjcerlighed og Sanddruhed for vore Djne . Israel begyndte ( efter Dom . 20. ) Krig med Guds Tilladelse , og bleve dog slagne ( 21. ) ; de spsrge Herren nok en Gang , og de faa det udtrykkelige Svar :  Drager op ! " Og de bleve nok en Gang slagne . Forst tredie Gang ( 28. ) vinde de Sejer . Denne Historie lcerer os , at naar selv efter en klar gubdommelig Anvisning Opfyldelsen af vor Begjæring ikke strax fplger paa , stulle vi dog ikke tvivle , ikke aflade , ikke forsage . Han hjcelper dog endelig sikkert . To Gangs maafis bliver vor Tro , som det synes , til Skamme , vor Forventning stusset ; og den tredie Gang felger Opfyldelsen paa det Herligste . Gud holder kun Gaven tilbage , for at vi stulle bede saa meget inderligere . At ) vidfte vi blot , hvor

<h4>1428</h4>Saaledes sukkede allerede Asaf i en haard Anfcegtelses Stund , og hvor Mange have ej sukket oa klaget paa samme Maade som han , naar den lovede Hjcelp , hvorom de bad og hvorefter de lcengtes saa saare , udeblev lcenge , ja saa lcenge , at den syntes dem slet ikke at ville komme , medens Noden trykkede dem saa lcenge , at det forekom dem , som om den aldrig vilde ophpre ! Hvorledes Hjcrrtet ved saadanne Prsvelser kan blive ligesom smderrevet , med hvilke smertelige Fornemmelser og sverre Tanker det kan blive ligesom gjennemboret,lcerer lobs Exempel , som klagede saa ynkelig :  leg har faaet Trcengsels Maaneder til min Lod , og de have talt mig nwjsommelige Ncrtter til . " I hele Timer eller Dage udholder Mennestet vel Herrens Hjemssgelser , men naar de vare Maaneder og Aar , da begynde vel endog Troshelte eller store Sjcele at forsage og at sige med Zion :  Herren har forladt mig . Herren har glemt mig , " ihvorvel Herren svarer :  Kan og en <b>Kvinde</b> glemme sit diende Barn , at hun ikke forbarmer sig over sit Livs Son ? Ja , sm hun end glemte det , saa vil Jeg dog ikke glemme dig ; ss , jeg tegnede dig paa begge mins Hcender " ( Es . 49 , 14 — 16 ) ; og Apostelen forsikrer :  Hvilke han forud beskikkede , dem har han og kaldet , og hvilke han kaldte , dem har han og retfcerdiggjort ; men hvilke han retfcerdiggjorde , dem har han og herliqgjort " ( Rom . 8 , 30 ) . Hvorfor forsager du da endnu , min Sjcrl ? Du v6d jo , at enhver af dine Anfcegtelser , enhver mork Time er bestikket ester den Eviges <b>vise</b> Plan og hsrer med til Rcekken af hans guddommelige Freds-Tanker . Fra Evighed af er allerede Timen bestemt , da din Anfcegtelse skal komme og da den stal gaa , og <b>Maalet</b> er afstukket , som Trengselens Velge

<h4>2004</h4>Hele Livet er en Taaresced : som Begyndelsen og Midten er , saaledes er ogsaa Enden , og Jorden er en Taaredal . Grcedende komme vi til Verden , som Vismanden siger :  leg udgav den fprste Rpst , ligesom alle Mennesker , med Graad " ( Visd . 7 , 3 ) .  Naar et Barn grceder Fødselen , " siger Augustinus ,  er det en Profet om sin tilkommende Jammer . " Ak ja , hvor Meget er der ikke at begrcede i Livet ! For ikke at tale om Angerens , Bpnnens , Kjcerlighedens og Medlidenhedens Taarer , som Guds Born fcelde her nede , hvor mange Angest- og Kors-Taarer l < e < be cj ned ad deres Kinder ! Kommer Sjcelen i <b>aandelig</b> Nod og Anfcegtelse, saa maa den klage med David :  leg vceder min Seng den ganske Nat , jeg blader mit Leje med mine Taarer ; min Graad er min Fpde Dag og Nat " ( S . 6 , 7 og 42 , 4 ) . Bliver Legemet angrebet as Sygdom og Smerte , begynder man snart , ligesom Ezekias , at grcede og at vaande sig ( Es . 38 ) . Sksr det da med dem , som ere Kjso as vort Kjsd , Mand , <b>Kone</b> , Barn , Fader , <b>Moder</b> , Broder , Soster , at de enten kastes paa Sygdoms haarde Leje eller bortrives af Dpden , hvad for en Taaresced fremkommer da ikke ! Da begrceder mangen Jakob sin Nakel , mangen Noomi sin Elimelek, mangen David sin Absalon , mangen Josef sin Jakob , mangen Martha eller Maria sin Lazarus . Lider man Vold og Uret paa Gods og 2 Ere , saa gaar det , som Salomon klager :  leg vendte mig og saae alle de Undertrykte , som ilde medhandles under Solen , og ss , der var Graad af de Undertrykte , og Ingen , som twstede dem ; der var Vold af deres Undertrykkeres Haand , men ingen Troster hos dem " ( Prced . 4 , 1 ) . Det er Livets Midte , isoer hvad de sande Kristne angaar ; de maa grcede og sukke , medens Verden glceder sig ;  Gud bespiser dem med Taarebrpd og giver dem et tredobbelt <b>Maal</b> af Taarer at drikke " ( S . 80 , 6 ) . Omsider

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015060948119'target=_>Brun, Johan Nordahl, 1843, Johan Nordahl Bruuns Hellige Taler</a><h4>1006</h4>af Sygdomme . Tre Slags nawnes i Terten : » ) Blinde , d ) Halte og e ) Visne , og lettelig lader sig Menneskets aandelige Trcmg henfore til disse trende Classer . n ) Blinde . Du ynkes , min Broder ! nåar du seer en saadan , du tcenker dig hans lange Nat , hans vaklende Trin , og hvor let han snubler over den mindste Anstods ' Steen ; men end om du greb ind i din egen Barm , maaskee var Vankundighed din <b>Sjels</b> Sygdom . Du kunde vcrre klog nok paa al den Deel og Ting , du foretog dig i dit jordiske Kald , og endnu ikke vide Saligheds Vei , og hvad vidste du da ved Veiens Ende ? Da , om din Vankundighed har vceret forsettlig, da , om du havde foragtet det Lys , du kunde faae , da blev det din Lod , at udkastes i det yderste <b>Mor</b> » ke , hvor der er Graad og Tamders Gnidsel . Kjendte du denne din Sygdom , du vilde forskrcekkes , fik du blot Lys nok til at see det Morke , du gik i ; o ! da vilde du blive videbegjcerlig , da bad du : „ Herre ! oplad mine Dine , at jeg maa see de underlige Ting i din Lov ! " Eller : udsend dit Lys og din Salighed , som kan lyse og lede mig og fore mig til dit hellige Bjerg ! Naar vi tale til Eder , Brodre af den haederlige Bondestand ! om Vankundighed , saa t < enker ikke , vi bebreide Eder , at I ikke ere skrifckloge , tcenker ikke , at vi fordre af Eder denne alskens Ovlysning , som vore fjantede Skribentere ide senere Tider svadsede om ; nei ! vi bebreide Eder intet , men vi bibringe Eder kun eders Trang til at vide hvad som tjener til eders Fred , kjende Budene , Gud gav os , til at leve christelig , kjende den Gud , som gav os dem , og troe paa ham , for at doe salig , og at grunde denne <b>Kundskab</b> , denne Troe , dette Saligheds Aaab paa

<h4>1388</h4>han saae , at man ret snart skulde afklcede ham , og kaste Lod om hans Kjortel . Dette saae Jesus , den Alvidende; men stigt kunde de viseste Dodelige ikke gjettet . Saa pludselig Forandring <b>moder</b> sjelden Lcereren i Guds Kirke . Vi ville dog ikke forlade med vore Tanker denne Skueplads , uden at fore med os nyttig <b>Lcerdom</b> herfrå. Man gjor i vore Dage saa meget Vcesen af Folke«Meninger, Folke-Stemme , den almindelige Villie , eller Gud veed hvad man kalder det . Her er et stort Beviis paa , hvor lidet den siger , hvor meget den vakter . Vel har jeg for Heras sogt at bevise , at der skeer Ting , som man ikke kan forklare sig af Tingenes sedvanlige Lob og de menneskelige Tilboieligheders almindelige Gang ; men hvor man er nodt til at antage en Ondskabs Hemmelighed , som virker kraftelig . en ond <b>Aand</b> , som tillades at virke paa Menneskets <b>Aand</b> ; men ganske umiddelbar skeer det dog ikke . Den onde <b>Aand</b> maa have sine Nedskaber , sine Betjente , sine Haandlcmgere ; det er onde Mennesker , og disse Ord maae have nogen <b>Kundskab</b> sorud , nogen Indstydelse paa den ovrige store Mamgde afMedmennesker ; og saa var det her , saa er det allevegne . Jesus havde , endog paa denne Dag , sine hemmelige Modstandere i de ypperste Prcester , i Pharisceerne , i Folkets Bverster . I Dag Horte man ikke deres Rost , eller de maatte folge Strommen . Man skulde tcenkt , at de evig vilde blevet for svage til at omstemme det eenstemmige Folk , til at berove lejum den Pndest , der i alle Hjerter syntes i Dag saa befaestet ; men Modstanderne behove mindre Tid , end en Uge , for at nedbryde alt , for at dove alle Roster , for at faae frembragt af disse samme mange tusinde Munde dette

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009061503004'target=_>West, Charles, 1860, Fremstilling af Qvindens Sygdomme</a><h4>1134</h4>selv , tilligemed en forsigtig Anvendelse af Jernmidler , i Forbindelse med smaae lndgifter af Jodkalium , frembragte en gradviis Standsning af Menorrhagien , en betydelig Formindskelse af alle den Syges smertefulde Fornemmelser og en kiendelig Aftagen af Livmoderens Størrelse . Jeg troer , at i de fleste Tilfælde en lignende Behandling , i Forbindelse med en temporær Afholdelse fra Samleie , vil have en Bedring og , hvis den fortsættes længe nok , en fuldstændig Helbredelse til Følge . I det ovenfor af mig meddeelte Tilfælde kunde der , med Hensyn til den Syges Alder og de mange Aar , hun allerede havde været gift , ikke være Spørgsmaal om . Muligheden eller i det mindste Sandsynligheden af , at hun kunde blive svanger . Men en nogenlunde lignende Tilstand iagttager man undertiden hos yngre Koner og faa Maaneder efter Giftermaalet . Her vil det være Lægens Pligt om muligt at skaffe sig <b>Kundskab</b> om Ægtemandens mandlige Evner , et Punkt , hvorom han maaskee selv kan finde det hensigtsmæssigt at udtale en Formening . Man maa betænke , at det ikke alene er den gamle Vellystling , men ogsaa den <b>aandelig</b> anstrengte . Lærde og Manden , der i lang Tid har levet fuldkommen afholdende , som kan have svage Evner i denne Henseende . En saadan Person gifter sig med Ønsket om at faae Børn , og flere af de blandede Følelser , som paa een Gang komme med i Spillet ved alle de Gienstande , der angaae Avleforretningerne , foranledige ham til at giøre altfor hyppige Forsøg paa Samleie . Acten skeer ufuldkomment ; nervøs Ængstelighed driver ham til endnu hyppigere Forsøg , hvorved deres Udførelse bliver endnu ufuldstændigere , og naar ikke et Svangerskab snart efter Brylluppet heldigviis indtræder , bevirkes en Tilstand af vedholdende Congestion til Livmoderen , som fører til Organets Hypertrophie , og <b>Konen</b> bliver lige saa uskikket til Undfangelse som Manden til at avle Børn . Men jeg har skrevet nok om en Gienstand , som jeg helst havde

<h4>4450</h4>Til en <b>vis</b> Grad ere begge disse Theorier sandsynligviis rigtige , skiøndt Hudcyster ere fundne under Omstændigheder, der ikke synes at tilstede nogen af dem . I alle saadanne Sæksvulster kan man finde alle Hudens Producter regelmæssigt dannede , som om de vare voxede paa deres rette Sted : Haarene paa en aldeles normal Maade indplantede i Hudvævet , som er forsynet med Sveed- og Talgfollikler; og Tænderne liggende i forskiellige Udviklingsstadier i Tandsækkene . Vi skylde en tydsk Lseges * ) Skarpsindighed de lagttagelser , som have fiernet dette Slags Tilfælde fra det Vidunderliges Gebeet og have viist , at der ogsaa der hersker Methode og Orden , hvor en ufuldkommen Viden kun formaaede at opdage Naturens Luner . En anden af hans Landsmænd har bidraget meget til at fuldstændiggiøre vor <b>Kundskab</b> , og jeg vil i Korthed anføre de Resultater , hvortil han er kommen . Dr. Steinlein * * ) fandt ved Obductionen af en ung <b>Kone</b> , hos hvem der ved fire paa hinanden følgende Tapninger var udtømt 78 Pund Pus , at den høire Æggestok var Sygdommens Hovedsæde , medens den venstre indeholdt flere smaae Sæksvulster , hvoraf ingen oversteeg en Hasselnøds Størrelse . Hiin bestod af mange Sæksvulster , som , med Undtagelse af en stor Sæk , der indeholdt flere Pund Pus , vare Fedtcyster , der varierede i Størrelse fra et Hørfrø til en Svulst af 3 til 4 Tommers Giennemsnit . Fedtet var af forskiellig Beskaffenhed i de forskiellige Svulster , og i de ældre var det ofte blandet med Håar , sammenfiltrede til en Masse . Efterat alle de indeholdte Dele vare udtømte , viste den største Deel af Cystevæggene sig glat og glindsende , men der fandtes paa

<h4>4835</h4>af Størrelse som Ægget af en kalkunsk Høne og . have i Almindelig- hed deres Sæde i Hypochondrierne eller i Sideregionerne , idet deres nederste Rand sielden , stiger under Høiden af crista ilei . I de fleste Tilfælde er der kun een Svulst tilstede , men undertiden ogsaa to paa de modsatte Sider , der da for det meste i alle Henseender ere symmetriske . De lade sig i Almindelighed lettere forskyde indad mod Legemets Midtlinie end udåd , og i en større Strækning opad end nedad , saa at de undertiden kunne trænges op under de falske Ribbeen , men sielden ned til regio iliaca og aldrig ned i Bækken- rummet . De ere faste , dog ikke uden en <b>vis</b> Elasticitet ; deres Overflade er glat og regelmæssig ; ved Stethoscopet kan ingen Lyd opdages i dem ; ved Percussionen afgive de en mat Lyd , der kun forandres ved Tilstedeværelsen af en med Luft , udspilet Tarmslynge bag dem , da den kan blive halvklar . Et Tryk paa dem er smerte- fuldt , men Smerten , der er forbunden med en egen Fornemmelse , af Qvalme , pleier at gaae over , naar Trykket ophører . Undertiden ere de imidlertid Sædet for en bestandig nagende Smerte , som kommer uden Foranledning , kan vedvare i Timer , Dage og Uger , og ophører ligeledes uden Foranledning , skiøndt man som alminde- lig Regel kan paastaae , at den forværres ved Bevægelse , lindres ved Rolighed De fandtes enten tilfældigt ved Underlivets Undersøgelse i et andet <b>Formaal</b> , eller den Syge blev selv opmærksom paa dem ved den Smerte , de foraarsagede hende . De maae voxe langsomt , thi skiøndt jeg aareviis har havt Leilighed til at iagttage deraf an- grebne Syge , har jeg dog aldrig kunnet forvisse mig om , at deres Størrelse undergik nogen Forandring . Jeg kiender desuden et Til- fælde , hvor en Svulst af dette Slags havde været mere end tyve Aar tilstede hos en <b>Dame</b> paa tresindstyve Aar , uden at have for- andret sin Form , Størrelse eller Leie . Denne <b>Dame</b> er bleven undersøgt af afdøde Dr. Warren , af Astley Cooper og Benjamin Brodie , og det kan tiene som et Beviis paa det Mørke , der hersker med Hensyn til denne Gienstand , naar jeg fortæller , at enhver af > disse udmærkede Mænd havde en forskiellig Anskuelse om Syg- dommen , idet den Ene meente , at den udgik fra Mesenterium , den Anden ansaae den for en bevægelig Nyre og den Tredie for en Svulst i Æggestokken .

<h4>5520</h4>Antage vi , at der er taget tilbørligt Hensyn til alle disse Punkter , saa er det næste Spørgsmaal , der paatrænger sig os , om det Tidspunkt efter Forløsningen , da et Forsøg paa at helbrede Fistelen ved en Operation bør giøres . Jég troer , at en saadan ikke bør foretages tidligere end tre Maaneder efter Forløsningen ; thi Modtage-I'ighed for Betændelse er større- hos en <b>Kone</b> , som nyligt har aborteret eller født , end hos en Anden , medens ikke alene alle plastiske Operationer paa Kiønsorganerne ere forbundne med nogen Fare , men en Grad af local Betændelse, som er aldeles uden Fare for den Syge , dog kan være mere end tilstrekkelig til at tilintetgiøre Forventningerne af den heldigst udførte Operation . En anden Grund for Operationens Udsættelse er , at en Fistelaabning sandsynligviis til en <b>vis</b> Grad vil trække sig sammen , og at det er godt at opnaae det fulde <b>Maal</b> af Bedring , som Naturen selv , kan bevirke , førend rnan 1 tåger sin Tilflugt til en chirurgisk Operation . Desforuden er det af yderste Vigtighed , at Vedkommendes Sundhedstilstand er saa god som muligt , forinden Operationen foretages , og man bør ikke beklage sig over nogen Udsættelse , der yder Leilighed til at styrke det almindelige Ve ] befmdende og forbedre Urinens Tilstand . Begge holde sædvanligviis lige Skridt med hinanden , og at operere paa en Tid , da Sundheden er svag og Urinen ' riig paa Phosphater , vilde efter min Overbevisning være at forspilde Muligheden af at gavne den Syge . Naar Aabningen ind i Blæren er forholdsviis lille , saa at Urinen i visse Legemsstillinger kan holdes omtrent en Time , kan man endnu opsætte Operationen een eller to Maaneder , forudsat at Blæren taaler Catheterens gientagne Indbringelse , en Operation , som den

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008110503018'target=_>Sand, George, 1845, Horace</a><h4>620</h4>Hun forstjonncdes paasaldende . Endnu altid mclank ? lff , havde hun ikke lcrnger dette Udtryk af Modlsshed , som Blaocriet girer. En ny Patroncsse sad allerede dag Hr. Poissons Diskz fra denne Kant havde hun allsaa Intet at bcfrygte . Hver Ssndag nsd hun med os den friske Luft , og hcndcs Suiidhcd , der lcrnge hcivde vcrrct alvorlig rystet , styrkedes sicnsynlig ved den simple , sunde Dicrt , jeg foreskrev hen ^ e , og som hun fulgte , uden at <b>vise</b> ecn eneste af disse ( Zapricer , der ere saa almindcli ' . , e hos ncroeuse <b>Fruentimmer</b> . ' Hr ndcs Ncrrvcrrelsc lukkede sagtens flere afmine ungcVcnner til Mansardcn , end tidligcrerc havde vcrrct Tilfcrldct ; men Eugenie paatog sig at tilbagcviscdem , hvis Interesse var alifor synlig improviserct , og hvad mine ncrrmcre Vennrr angik , tilgave vi dem af ganske Hjerte , ar de viste sig en Smule mere empresscrede end fsr . Disse Samlinger af Studenter , hvis livsfriske Ungdom , naar de vare paa deres egen Haand , aabenbarede sig pan en saa vilo og energisk Maade , og som , for at behage fire qvindcligc Vcrsncr , hvis Dyder imponcrede dem , under vort Tag pludselig bleve stille og sindige , var Noget , der i og for sig var rorcnde og smukt . Alle felte sig vel . Horace blev mindre egoistisk . De unge Mennesker sik Smag paa « Mere Scrdcr , end de , de i Almindelighed havde for Die . Marthe glemte fin ssrgcligc Fortid . Suzanne loe af gansse Hjerte og udvidcde sine Ideer . Den eneste , der ikke drog nogen moralsk Gevinst af disse Samlinger , var Louison ; men hun lcrrte i det mindste atbeherffe sig . Owcndskjsnt hun aldrig ophorte at verre Nigorist i Ordets strengeste Forstand , var det hende aldeles ikke übehageligt at behandles som <b>Dame</b> af unge Mennesker , hvis <b>Locrdom</b> og <b>Dannelse</b> forekom hende colossal .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2007091703001'target=_>Wergeland, Henrik, 1853, Henrik Wergelands Samlede Skrifter</a><h4>165</h4>Man har ventet Alt i Henseende til Almuen , af Religionen. Sandelig , den gjor vel det Bedste , men ikke Alt . Den danner vore Hjerter , vore Sæder ; den giver voreHoveder den vigtigste <b>Kundskab</b> : den om Gud , og anden dyrebar Viden ; men endda bliver der Rum i vore Hjerner tilovers for den verdslige Viden ; ja Religionen opliver vor Stræben efter denne ved Udødeligheds-Haabet . Skulle I nu ingen anden <b>Kundskab</b> faae i Eders hele Liv , I Mænd og <b>Kvinder</b> , end den Religions-Underviisning, I fik som Barn , saa forbleve I , Eders hele Liv igjennem , i <b>Sjels</b> Henseende ikkun Bern , ikkun Umyndige ! Hører her , hvad en Hædersmand , der redeligen arbeidede for Eder , henfarne Biskop , Doctor Hansen i Christianssands Stift , har sagt for denne vor Sag :

<h4>182</h4>bedre Væscn — at den fortjener ikke at gaae udmajet , som ikke kan eller vil læse , eller lære sine B # rn eller yngre Sedskende at skrive ; og da , er der Een , som har erhvervet sig saameget Vid , da vil hun indsee : at Tarvelighed , at alvorlig Sædelighed i tarveligt Klædebon , er den Dyd , som det anstaaer Norske <b>Kvinder</b> at ove og at <b>vise</b> ! — at der ingen Ære er i at efterligne Fornemmeres og Rigeres Daarskaber og Forfængelighed! Seer , mens I spinde , ville Mændene ved Arnen fortælle Eder , hvad de have læst , og Jæse for Eder . Af dem skulle I lære <b>Kundskab</b> ; I <b>Kvinder</b> , lærer dem Dyder til Vederlag ! — Ja , faaer Smag paa den cedle Forndjelse , det forskuffer at samle <b>Kundskaber</b> : da har den eenlige Stue , Huusligheden, Hjemmet faaet nye Tillokkelser , da vil aldrig Tiden falde Eder lang i de for den Ørkeslese alt for lange Vinteraftener; ja da har ogsaa Selskabeligheden , Eders Omgang med hverandre faaet nye Tillokkelser , idet store , Sundhed og Formue edelæggende , Gjæstebude tabe deres . Digteren Rein siger :

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015061048103'target=_>Brontë, Charlotte, 1850, Shirley</a><h4>765</h4>hende , og siden havde hun ikke seet hende . Hendes Fader dode i en temmelig ung Alder , og allerede i adskillige Aar havde hendes Onkel , Prcessen , vcrrc ' t hendes eneste Beskytter . Han var , som vi have seet , hverken af Naturen eller ved Vane stikket til at bcrre Omsorg for en ung <b>Piges</b> Opdragelse ; den brod han sig kun lidet om , og han vilde sandsynligviis aldeles have forssmt den , hvis hun ikke selv var bleven angest for at vore op i altfor stor Uvidenhed , og stundom havde bedet om lidt Vejledning og om Midler til at erhverve sig saadanne <b>Kundskaber</b> , som ikle kunne undvcrres . Imidlertid havde hun dog en ned » trykkende Folelse af , at bun stod langt tilbage , at de <b>Kundskaber</b> og Færdigheder , hun havde erhvervet sig , vare langt feerre end dem , som <b>Piger</b> i hendes Alder og Stilling i Almindelighed besad . Med Gla ' de benyttede hun sig derfor af sin Kousine Hortenses venlige Tilbud , som denne gjorde hende kort efter sin Ankomst til Hollow Molle , at la ' re hende Fransk og fiint Broderi . Mademoiselle Moore fandt Behag i denne Beskjeftigelss , fordi den gav hende c » <b>vis</b> Vigtighed ; hun holdt af at hovmesterere sin lærvillige , stjondt lidt flygtige Elev . Hun betragtede

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112101023'target=_>Skaar, J.N., 1880, Norsk Salmehistorie</a><h4>11499</h4>er fpdt paa Bragerncrs i Drammen den 10. September 1748. Hans Forceldre var Major ved ftrste akershusiske Regiment Johan Kristian Vibe ( - f - 1764 ) og Anne Katrine Pihl ( f 1773 ) , en Datter af Provst i Gudbrandsdalen loakim Pih l . 1766 blev han Student ved Københavns Universitet , og var en Tid Ove HBegh-Guldbergs Privatsekretcrr , hvorefter han 1778 blev Kammersekretnr , men d ^ de ien ung Alder 1782. Mest bekjendt er han som Visedigter og som en af de betydeligste Personligheder i  det norste Selstab " , hvis Sekretoer han var . Selskabets bekjendte Vertinde , Madam luel ,  Vibes gjestmilde Pleiemoder i hans sidste sørgelige Dage , og hos hvem han udaandede sit sidste tunge Suk " , l ) kaldte ham — iModscetning til hans sarkastiske Broder , loakim Kristian , —  den skikkelige Vibe " . ForMigt var hans Liv ikke uden store SlrKeligheder , hvao han selv ikke loegger Skjul paa . Se Slutningen af hans Afstedsode i det norske Selstabs Versprotokol . Efter hans Dpd udkom ( 1783 )  De nysgjerrige Mandfolk , en Komoedie " . Han stal have strevet den efter Opfording af Dronning luliana Maria , som ikke kunde finde sig i , at Nysgjerrighed alene blev tillagt <b>Fruentimmer</b> . Brandt og Helveg tillcegge ham Forfatterskabet af Nr . 9 i G ul d bergs Salmebog :  Syng HM , min <b>Sjel</b> , vcer hjerteglad " , en fri Omarbeidelse af den gamle :  Et lidet Barn saa lysteligt " . Men Rah bek meddeler i sine Erindringer ( 4 , S . 12 ) , at Vibe har til den guldbergske Salmebog leveret to eller tre deilige Sange . Da der foruden Nr . 9 ikke er andre end Nr . 93 og 312 , som han kan have digtet , er det sandsynligt at disse , nemlig Stemmer alle op at sjunge ( Landst . Nr . 173 ) . Saa Icrnge du har <b>Maal</b> og Mund ( Landst . Nr . 78 ) ere forfattede af ham . Som Forfatter af den ftrste har man vistnok

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008092203009'target=_>, 1858, Bibelen, eller Den hellige Skrift</a><h4>213</h4>ret lykkes . Men Jordarbeidet er her kun det dcngang eneste Erempcl paa Menneskets ydre jordiske Kald ; paa enhver Stand , paa Ethvert Kald , enhver Beskjeftigelse her paa Jorden hviler en Forbandelse , d . e . megcn Nod og Trengsel , megen forgjeves Anstrengelse endog under det moisommeligstc " Arbeide erfarer Enhver i sit Kald ; dette er det udvortes Onde som Syndens Straf . Ved denne ssorbandelse er Jorden blevcn til en Taarcrs Dal , saa at Alt under Solen er forfengeligt ( Pred . 1 , 2. 3. 14. ) . og Mennesket nu alene stal tragte efter den uforgjengelige og uforvisneligc Arv i Himlen ( iPet . 1 , 4. ) . Det forstaacr sig , at " dette alene er den ene Side af det timelige Onde , og at Mennesket i Naadens Tid og formedelst Naaden ved sin Omvendelse kan forvandle til det gavnligste Tugtemiddel , det kosteligste Legemiddel just det , som for det i Synden blivende Menneske er . en Straf , og vel endog kun en Forsmag paa tungere , evig Straf . — Siden denne Tid er nu ogsaa i Naturen foregaaet en stor Forandring; den stiller nu imod Menneskets Herredomme en Modstand , som dets Viisdom og Kunst altid kun i ringe <b>Maal</b> formaaer at overvinde , og som det ogsaa kun da ret overvinder , naar Forlosningens Velsignelser udbredc sig over hele Folk og Lande . — Den anden Mennesket forkyndte Straf er Doden , med Alt , hvad der gaacr forud ( Sygdomme og al deraf opstaaende Nod ) og felger efter . <b>Sjelens</b> evi ^ e Fordervelse . Den legemlige Dod er det synlige Afbillede og Forvillede for den evige Fordervelse , det dunkle Forheng for den hiinsidigc Verden , som for den uomvendte Synder kun dolger haablos Elendighed ( om Betydningen af , , Dod " smlgn . C . 2 , 17. Anm . ) — Eva ( Heva , egentlig  Chavvah " ) ( V . 20. ) betydcr , , Liv " ( deraf det greske Navn Zoe ) . — Det synes , som om der til Grnnd for dette Navn , som Adam just nu giver sin <b>Hustru</b> , ligger haus Tro paa den ( V . 15. ) givnc Forgjettelsc om 3 Nenneskeslegtc ' ns Opholdelsc og fremtidige Gjenoprcisning ved <b>Kvindens</b> Sed .  Omend domt til Doden , vil " Menneskcstegten dog ikke gaae under , men seire over ham , der forforte den til ' Synd og Dsd , ' ved <b>Kvinden</b> . " — Kjortler af Skind ( V . 21. ) synes at forudsette Drabet af Dyr ; Anviisningen hertil blev vel altsaa given Mennesket af Gud , og lagde Grunden til de blodige Offere . — - Hvad tidligere Mennesket i den forstc Skamfolelse efter " Faldet selv havde gjort , det gjor nu Gud paa en endnu fuldkomnerc Maade ; han bekrefter , som sin Anordning , Folelsen af Skam og den deraf ndspringende Tugt og Wrbarhed . Dette er den forste Begyndelse til Guds Lovhuusholdning , i hvilken Gud forelobig ved ydre Anordning tilbagedriver og svekker Lysten til det Onde , forskaffer sin Villie en ydre Anerkjendelse iblandt Menneskene , og vekker og opholder i dem en <b>Bevidsthed</b> om Synden , - uden at dog allerede ved disse Anordninger en inderlig Overvindelse af Synden skulde vere mulig ; men han vil ved en saadan Oftdragelse lede Menneskene til den fuldkomne Seier over Synden og den fuldkomne Opfyldelse af Loven i Christo Jesu . — I V . 22. gjengives Grundtcxtens dll ^ k vistnok rettere med  er blevct , " end med  har veret . " Men naar Gud saaledes siger :  See Mennesket er blevet som en af os til

<h4>1755</h4>er enhver den Andens Nceste d . c . ordret : en Staldbrodcr , Kammerat , Ven , fordi Alle staa i det samme Forhold til Gud . Det forste Vud i den anden Tavle taler om Krcenkelse afNcesten derved , at man forgriber sig paa ham selv , hans Person : det andet og tredie om Krcenkelse i Gjerning , med Hensyn til hans <b>Hustru</b> , hans Eiendom ; det fjerde om Krcrnkelse af Nersten ved Ord ; det femte ved den onde Lyst . I alle disse Bud — dette er overordentlig mcerkvcerdigt , naar vi dermed sammenligne de hedenste Love — er enhver Pligt umiddelbart hcnfsrt til Gud , udledet af Forhold til ham , intet Spor findes den af Trudsel med borgerlige Straffe , af en Benyttelse af Lydighedcn mod Gud til timelige <b>Formaal</b> ; tvertimod , idet denne Lov dannede Kjernen af Folkets hele Forfatning , skulde det fremlyse aldeles klart deraf , at alle andre Vud bavde den Bestemmelse at gjennemfore de ti Bud i ethvert Livsforhold . Ligesom derfor Lovgiveren er hellig og en <b>Aand</b> , saaledes maa alle disse Bud opfattes som hellige , rettede mod det Indre , scedelige , aandelige , intet som blot udvortes . Vistnok skrider Forbudet frem fra det :  Du stal ikke staa ibjel " til det :  Du skal ikke begjcere " , og i de tre forste Vud af den anden Tavle er der fortrinsviis Tale om Gjerninger . Men idet der fra Forbudet gjores en Overgang til Begjceringen , viser det sig deri aldeles klart , at heller ! lke Mordet maa opfattes paa en udvortcs Maade , men at det i sin inderste Rod , i Hådet , Vreden , Misundelstu , er Gud en Vederstyggelighet » , ligesom enhver ukydsk Gjerning ogsaa udenfor Wgteskabet er mig forbudt som en Begyndtlse til Wgleskabsbrud , som ben forste Utring af ben onde Vegjcrring , ligesom Havesygen , ogsaa for den bliver til Tyvene , forbydes , som dettes indre Grund . Den groveste Overtredelse af Budet , ncrvneS udtrykkelig , fordi det ved den efterhaanden lod sig tydeliggjsre , hvorledes enhver anden ringere eller indre var med indbefattet deri , medens Forbudet af mere indvortcs Synder , som Grun » den til de grovere , paa langt ncer ikke vilde havt det Levende , Sandseligtglibende ved sig . Udviklingen af de ti Bud i de fslgende Love viser tydeligt ogsaa for den groveste Forstand , hvor dybt de maa opfattes . Saaledes gav Herren i disse  ti Budord " et raat Folk , der nersten var hleven hedensk , de ftlste Grundtrcek af en hellig Lov . som selv Guds eenbaarne Son ikke har oplost, men opfyldt , som heller ikke Han er gaaet udover , da ban med den Mosaiske Lovs egne Ord anfsrer de to fornemste Bud , i hvilke den hele Lov og Profeternc hang , og som danne Grundlagtt for begge Tavlerne ( Math . 22 , 37 — 39. ) , og ogsaa i Bjergprccdikenen tåger han kun bort de Skranker , som endnu omgav den i Loven selv givne Kjerne for Umyndighedens Tid . — Man adstiller i Loven scedvanlig Moral- eller Scedeloven fra Ritual- ( eller Ceremonial- ) og den borgerlige Lov , og antager , at de tyende sidste Dele af den Mosaiske Lov ere blevne ophcevcde ved Christendommen , den fsrste derimod ikke ; og denne foisle skal ba vcere indeholdt i de ti Vud , som derfor have fuld , blivende Gyldigbed lige indtil i de mindste Enkeltheder , ogsaa for de Chrlstne . Men en saadan Adskillelse antydes nu ingensteds i Loven selv . Ogsaa i be ti Bud i Sabbatsloven er aabenbart inbeholdt noget Rituelt , og ligesaa er deri udtalt Giundlaget for Israels borgerlige Lovgivning , som bestaar i Folkets Pag » tesforhold til Jehova , dets Konge . Omvendt er Ritualloven ikke fuld af vilkaarlige, tilfcrldige Forskrifter , men alle Bestemmelser i ben ere Legemliggjo«lser af de Forholde , som have deles Grund i de ti Bud ; og netop i Israels ! 6 »

<h4>2653</h4>V . 19. Ofrene skulde bringes af de Dyr . paa hvilke Folkets bele ydre Tilvcrrelsc bcroede , af de vcrrdisuldeste , rene Huusdyr . Blandt disse skulde iscer Havdyr ofres , som de , der vare de stcrrkere og fuldkom , ! ere , med Undtagelse af Takofrcne ( 5. 3 , 1. — At denne Uudtagclse ikke auforcs her , kommer deraf , at Takofrct ikke horte til de  scrrdeles hellige . " til de egentlige vigtigste Ofre . — De « frivillige 23 Ofre " vare ringere end Loftcofrene , fordi disse sidste beroedc paa en mellem Herren og Mennestet bcstaaende Pagt . Heller ikke for den ringeste Art af Ofre var nogen anden Lyde cftergivet end denne alene , at Dyret havde et Lem for stort eller for lidet . — lovrigt ere ogsaa disse Feil alle anfsrt erempelviis , hvilket fremgaar dcraf . at der i 5 Mos . 15 , 21 tilfoics Lambed . — De fire Udtryk :  fortrykt , 24 kmmst , sondcrrclictf afskaarct " sigte til de forskjellige Maader , bvor . paa man i Oldtiden pleiede at gilde Dyr . — Tillcrggct : s , oss I skulle ikke ssjore « ogct saadant i cdcrs Land " kunde man forståa om saadannc Dyrs Ofring ; men da indcholdt disse Ord kun en Gjentagelse af . det Foregaaende . Derfor forstaar den jodiske Overlevering det vel rettere om et Forbud mod ovcrhovedet at gilde Dyr . Grunken til dette Forbud er da ikke den , at den samme Fremgangsmaade med Mennesker derved desto sikrere skulde forebygges , men dcr « imod , som dets Lighcd med andre af samme <b>Formaal</b> ( C . 19. 19 ) , viser , at der stuldc indpmges en <b>vis</b> Wrefrygt for Guds Ekabning . som Mennesket vistnok skal beherske , men ikke forandre og forstyrre i dens Grundformer ; ogsaa dette har da fortrinsviis en sindbilledlig Betydning , i Lighed med Forbudet mod at behandle Mennesker paa samme Maade . ' Sml . 5 Mos . 23. 1. - Grunden til , at intet Dyr 27 var tjcnligt til Offer , for den ottcnde Dag , cfter at det var fodt , er uden Tvivl den , at det fsrst da begyndcr at blive spiseligt ; men heller ikke dette er anordnet for den virkelige Episens Skyld , men for Eindbillcdets Skyld . — Ved Oj ' c ( V . 28 ) forstaaes egentlig  en Ko ; " thi der kan naturligviis kun verre Tale om Dyrets <b>Moder</b> ; Oze er her Slcrgtsnavnet . Moderen og dens Affodning maatte ikke siagtcs paa samme Dag . Den gamle chaldcriskc Omskrivning gjor ber folgcnde Tillcrg :  Mit Folk , ligesom eders Fader i Himlen er barmhjertig , saa vcrrcr ogsaa I barmhjertige paa Jorden : en Ko og et Faar skulle I ikke stagte tilligemcd deres Unge paa samme Dag . " Det er da ligeartct med Forbudet i 5 Mos . 22 , 6. Heri ligger et Vink . - ligesom Gud ovholder Dyrenes Slcrgter og Arter i Ekabningen, saaledes skal Mennesket ikke udrydde dem , men kun bruge dem til Guds Tjeneste og sin egen Nytte . Men et Skridt til en Udryddclse vilde en saadan Drabsmaade vccre , naar den blev almindelig . — Tct ( Lovofrct ) skal cedcs Paa den samme Tag , som det helligste 3 n af Takofrene . Sml . C . 7 , 15.

<h4>3574</h4>Mos . 17 , 7. Anm . Om hvorvidt Afgudstjcnestcn var en Dyrkelse af de onde Aander see 1 Kor . 10 , 19. 20. Anm . — Til dc nye ( Guder ) , som vare komne noer fra d . e . hvis Dyrkelse de havde erboldt ved Nabofolkene , Moabiterne og Ammonitcrne , senere iscrr Canaaniterne i Phonicien , Syrerne . — Og blev vred for fine Son- 1 9 ners og Dettes Fortornelscs Skyld , blev uvillig af Vrede over sine Semncr og Dotte . — Her begyndcr nu Forkyndelsen af Strasscdommene over det ulydige Folk . — Thi de , de ere en faare forvendt Sloegt osv. ordreti thi en Forvendthedcrnes Slcrgt ( en lognagtig, bedragerist ) ere de , Ssnner , ingen tro ( fast , tilforladelig, provet ) iblandt dem . — Ved deres Forflrngclighcdcr , Intethedcr; dette Navn bcere Afguderne ogsaa ellers oftere . — Den i Slutningen af dette Vers indeholdte Spaadom er overordentlig mcrrkvcrrdig og danner Grundlaget for en lang Rcrkkc af paafolgende Epaadomme. Her forkyndcs nemlig for forste Gang . at Gud engang vil forkaste det kjodclige Israel og i dets Sted udvcrlge vcdningcrne til sit Folk , saalcdes som Paulus forklarende anfsrcr disse Ord Rom . 10 , 19. Israel har udfort Guds Nidkjarhed ved en dets crgteskabclige Pagt krenkende Dyrkelse af fremmede Guder ; nu tåger den krcrnkcdc Wgtehcrre sig en anden , ringere <b>Hustru</b> og vcrkkcr dermed paa sin Side sin Husttues Iversyge . Disse Hedninger ere Me et Folk , " fordi kun Folket under den sande , evige , almcrgtigc Konge i Ordcts fulde Forstand er et Folk . De ere daarligt Folk , " thi da de boldt sig for <b>vise</b> , eie de blcvnc til Daarer , de have hengivet sig til det taabeligste Afgudcri . Norn . 1 , 22 ff . Dette sigter ligcsaa meget til Magtcn , Lykken , Seiren over Israel , bvilken Gud forlener Hedningerne . som ganjkc iscrrdeleshed til Guds Nigcs Nddrcdelse , Stiftelsen af Naadepagten med Hedningerne . Sml . Iversygc Lue . 15 , 28. Ap . Gj . 14. l flg . 13 , 45. — Oss den ( Ilden ) ssal brwudc til nederste Helvcde . Ikke blot med fordigaacnde , jordiske Straffe vil Gud hjemsoge dem , men endog med fnldkommen Tilintetgjorelse . De i disse og dc fslgendc Pers indcholdtc Trudsler ere allerede udtaltc Moses tidligere Taler 3 Mos . 16 , 14 ff . 5 Mos . 28 , 15 ff . — Fiendens Fortornelse er ( sml . V . 19. ) Vrede over Fienden ; Gud vil ikke fortorncs over Fiendcrnes Hoffcrrdighed . Meningen af dette stcrrkt menneskelig udtalte Sted er : Det hoiestc <b>Formaal</b> ved Folkenes Ledelse er Guds Wre ; ogsaa ved sine frafaldne Berns retfcrrdige Afstraffclse sorger han for at de Ugudelige ikke skulle rose sig af de af ham beskikkede Etrassedomme , som om det var deres egne Gjerninger . Sml . dc lignende vigtige Etcdcr 2 Mos . 32 , I < ! sa > r ( ss . 10 , 8 - 19. Ez . 20 , 9. 36 , 11 fl. At ikke deres Fiender sknldc holde sig fremmede derover . Luther : dlivc stolte . — Thi dc , dc ere Folk , som sordoervcs ved ( sine egne ) Naad ; Luth.i hvori intet Raad er . Fra nu af angiver dcn hellige Sanger ncrrmere Aarsagcrne til at saadanne Strafftdomme ramme Israel . Fordi de af Telvforblindelse lobe i Ulykken , fordi Gud selv har overgivet dem dertil , men netoft derved ogsaa stedse mere udsondret sine sande Tjenere fra det udartedc Folk / netop derfor er <b>Maalet</b> for

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016111429005'target=_>, 1837, Huus-Postil, eller En Samling af forskjellige Forfatteres, meest Luthers, Franckes og Pontoppidans, Betragtninger over Søn- og Fest-Dages deels Høimesse-, deels Aftensangs-Texter, samlede til en Aargang, og udgivne i den Hensigt, at befordre og vedligeholde christelig Huus-Andagt</a><h4>459</h4>styggelighed . Deraf kommer det , at mange , nåar de komme i Uenighed med deres Næste , da holde de sig fra Sacramentet , og ei ville bede et Fadervor ; thi de Ord stode dem for Panden , som de bede , nemlig : forlad os vor Skyld , som vi og forlade vore Skyldnere : i Tanke , at saasom de ei forlade , de da stulle afsige Dommen over sig selv , at Gud ei stal forlade dem . Det er og sandt , man tor eiheller tænke andet ; vil du ei forlade , saa vil Gud eiheller forlade dig . Betamk derfor vel , hvad for et Bad du tillaver dig , nåar du beholder og ei lader Vreden fare . Du gjor ei heller din Sag bedre med det , at du ei beder , og ei vil gaae til Sacramentet , men kun vcerre . Thi Gud dommer dig i den Stand , han sinder dig . Fola derfor Christi Raad ; ydmyg dig for Gud , og bekjend dine Synder ; forson dig med din Næste , og lad Breden fare . Ofre da din Gave : c . ; thi ellers horer du , at Gud ei stjotter om dig , og ei vil behage dit Offer og Bon . Det mener Christus , nåar han siger : lad din Gave blive paa Alteret . Som han vilde sige : du udretter dog intet dermed hos Gud . Det er det ene Partie , som giver Aarsag til Brede , og som foruretter sin Næste ; de stal ei alene ydmyge sig for Gud , og bekjende , at de have gjort ilde , men endog for deres Næste , og saa kan de vente Forladelse af Gud . Det andet Partie er , som forurettes , og mener sig at have Aarsag til Brede . Dem paaminder og Herren her , at de stal tilgive og ei længe lade sig bede . ' Thi de ere og , siger Herren , i stor Fare . Thi dersom du lange lader dig og ei er villig til at lade dig forsone , saa vil din Motstander befale Dommeren , Gud i Himmelen , Sagen , og sige : Herre ! jeg haver gjort det , jeg stulde , hos dig , sinder jeg Naade , men ei hos Mennestene ; velan ! jeg vil befale dig Sagen . Dersom nu Gud vilde overfalde dig , hvad mener du da at din Dom blev ? dig , som ei vil tilgive og forglemme , dig vil han tage , og antvorde Tjeneren dig , og du stal kastes i Fomgsel . Der stal du ei komme ud af , forend du betaler den sidste Skjaerv . Det er , du stal ingen Naade sinde , som han siger i Luc . 6 , 38. : med den Maade , somImaale med , stal eder igjen imaales . Msaa vil Herren have af begge Partier , at man stal vare barmhjertige , lade Breden fare , og va-re venlige mod Enhver . Thi ellers anklager det femte Bud os , at vi ere Manddrabere for Gud , og altsaa ml Straffen ei blive borte . Denne <b>Lærdom</b> og Levnet stal blive blandt de Christne , og flittig oves . Ellers behager det ikke Gud , og vi gjore os selv ei alene Skade paa Legem og Gods , men endog paa <b>Sjelen</b> , hvormed Herren truer . Men udenfor Christi Rige i verdslige Embeder og Rettergang , der haver det femte Bud ei saaledes sin Gang . Der haver Gud givet en særdeles Befaling , som man stal folge og efterkomme . Thi den , som er i det verdslige Regimente , haver Befaling , at han stal blive vred , straffe og staae ihjel , naar Undersåtterne begaae noget Übilligt , og det , som er Døden værd . Ligeledes haver Fader og <b>Moder</b> i Husene en scerdeles Befaling af Gud over Born og Tjenestefolk , at de ei stal lee , naar Bornene og Folkene førsee sig ; men de stal sættte dem i Rette og straffe dem . Det befaler Gud dem , og dersom de ei gjore det , saa ere de Gud

<h4>992</h4>herom , er og strevet os til Advarsel , og omendstjondt Ingen er , har varet , eller maastee torde ventes i vore Tider , og hos os ister , som Evangelii Lys ffinner saa klarligen iblandt , der stulde virkeligen udgive sig for at være Christus, saa have vi dog virkeligen hoi Aarsag til at vogte os for en falsk Christus. Hvorledes da ? Er det ikke en falst Christus , naar denne sande Christus , som har aabenbaret sig for os i Evangelio , ikke saaledes erkjendes , læres og troes , som han har aabenbaret sig for os , men man forvender den <b>Lærdom</b> ikke alene om hans Person , og negter hans evige Guddom , og at han er den sande Gud over Alting , velsignet i Evighed , Rom . 9,6 . : c , men og om hans Midler-Embede , naar man negter hans Forsoning og Fyldestgørelse , som er den eneste Grund til vort Haab , ja til al vor Retfoerdighed , Liv , Kraft og Salighed ; og at vi stulle blive retfærdiggjorte uforskyldt af Guds Naade ved den Igjenløsning , som er i Christo Jesu , hvilken Gud haver fremsat til en Naadestoel formedelst Troen i hans Blod , til at <b>vise</b> sin Retfoerdighed for de forgangne Synders Forladelse udi Guds Taalmodighed ; til at <b>vise</b> sin Retfoerdighed paa den noerværende Tid , at han maatte være retfcerdig , og retfoerdiggjore dem , som ere aflesu Troe , Rom . 3 , 24 - 26. Er det ikke en falsk Christus , naar man troer , at Christus er kun kommen til at forlade Synderne , men ikke tillige til at rense os fra al Uretfærdighed , 1 Joh . 1,9 . og at forlose os fra al Uretviished , og rense sig selv et Ejendomsfolk , som stal være nidkjært til gode Gjerninger , Tit . 2,14 ; naar man vil bruge Christi Fortjeneste , og den Guds Naade , som os i ham tilbydes , til Uteerlighed , lud . V . 4 , og til Ondstabs og Hykleries Skjul ? Er det ikke en falst Christus, naar man ikke ligesaavel vil have Christum til sin Prophet og Loerer , til sin Konge og Regjerer , som til sin Ypperstepræst og Forsoner ; ligesaavel have ham til Viisdom , Helliggjørelse og Gjenfodelse , som til sin Retfoerdighed? 1 Cor . 1,30 . Er det ikke en falst Christus , naar man negter , forkaster og forhaaner Jesu hellige <b>Loerdom</b> med Ord eller Gjerning ; naar man mener , at man kunde nok være hans Discipel , uden at hade Fader , <b>Moder</b> , <b>Hustru</b> , Born , Brodre , Sostre og tilmed sit eget Liv , Luc . 14,26 , uden at førsage alt det , man har , 35. 33 ? naar man icenker , man kunde nok komme efter Christum , uden at fornegte sig selv , tage sit Kors op daglig og folge ham , Luc . 9,23 ; man kunde nok være den Hoiestes Barn , uden at elfle sine Fiender:c. Matth . 6,44 - 48 ; man kunde nok tjene og behage baade Gud og Mammon, baade Gud og Verden , fljondt Christus siger , det er umuligt , Matth . nem den Knevre Port og paa den trange Vei , Cap . 7,13 - 14 ; man kunde nok komme ind i Himmeriges Rige , uden at gjore Faderens Villie , fom er i Himlene , V . 21. fg . ; man kunde nok være en klog Mand , og havde derfor ikke bygget sit Huus paa Sand , eller bedraget sig i sit Haab om Saligheden, endstjondt man ikke gjorde efter Jesu Ord og Lærdomme , V . 26. fg . ; man kunde nok arve Guds Rige , omendstjondt man imod Christi og Hans

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013031324028'target=_>Holberg, Ludvig, 1857, Ludvig Holbergs trende Epistler til ***</a><h4>1457</h4>Aand , og blive giorte til blotte Machiner . Ingen <b>Lcerdom</b> kan synes mere Paradox ; men den blev af mange med Begierlighed antagen , efterdi man holdt den for det eneste Middel , for at hindre , at Menneskers Sicele ilke stulde confunderes med Dyrenes , hvoraf en Folge blev , at de ene saavel som de andre enten maatte vcere materielle , eller bcgge aandelige og conseqventer udodelige . Derimod have mange giort sig Umage med at igiendrive denne <b>Lcerdom</b> , betienende sig as Argumenter , som tages af Dyrenes Qvaliteter, efterdi man hos dem finder Gloede , Bedrovelse , Begierlighed, Afsky , Venstab , Had , Ihukommelse og andet , som ikke kan tillcegges Machiner . Ja nogle have gaaet saa vidt , at de hos Dyrene ogsaa have fundet Tegn til Religion. Dette uanseet have dog Cartesianerne hceuget ved deres Mening , sigende , at , hvis man tillcegger Beesterne Kundstab , pdelcegger man alle de Beviisligheder , som kan anfores om de menneskelige Siceles Udodelighed ; og at de , som soge at bestride denne deres <b>Lcerdom</b> , kan ansees , som Epicurceere , hvilke , for at opofre Religionen , gipre Menneskets Sicele lige ved Beesternes . Disse derimod soge at befrie sig for saadcmne Beskyldninger , ved at statuere en Forstiel l6M6i-6utiam Zpecitieamj imellem Dyrs og Menneskers Sicele . Men Cartesianerne holde for , at meer og mindre loser ikke Knuden ; efterdi deraf ikke andet flyder , end at Dyrs og Menneskers Sicele differere allene udi meer og mindre Fuldkommenhed : ja , der findes de , fom <b>vise</b> , at jo storre Qvaliteter , der antegnes hos Dyrene , jo mere styrkes Cartesii <b>Lcerdom</b> derved . For Exempel : En Fugl , mod Tiden at den vil fode , laver med Konst og Behcendighcd til et Rede , fom kan vcere beqvemt for Ungerne ; da en <b>Pige</b> derimod , der om ingen Fpdsel haver hprt , og ingen Barsel-Seng seet , vilde bilde sig ind , at hendes Frugtsommelighed ikke var andet end en Hcevelse , som foraarsagedes

<h4>1826</h4>og medens jeg « ceder mig , bander min Skrceder : c . Det sidste skeer , efterdi mine Klceder ere saa trange , saa at det er fast det samme som at fpre sig i Harnist . Mig synes , at Verden er trang nok , saa at man ikke haver nodigt at giyre ven trangere . Du vil uden Tvivl hertil svare : hvad kan Skrcederen gipre dertil , at Klcederne blive saa umagelige og piinagtige ? Han gior jo ikke andet end hvad Folk vil have . Intet er vissere , hvorudover , naar jeg tcenker mig ret om , saa haver jeg heller Aarsag til at vcere vred paa mig selv , der efter Moden bestiller Klcederne , end paa Skrcederen , der dem , eller rettere paa Moderne , der tyrannisere baade over Skrcederen , over mig og over alle andre , og det udi saadan Grad , at det holdes for en Egensindighed , ja hartad en Rebellion at scette sig derimod , hvorudover man maa med Taalmodighed stikke sig derudi , som udi andre Uleiligheder, hvilke man ei kan hindre , og sige med Bonden hos ? 6 tranmm , der havde mistet sin brogede So : mo6o sie , mo6o Bic . Vi maae takke Gud , at de Parisiske Skrcedere, som fabriqvere vore Klcede - <b>Moder</b> , ikke foreskrive os det , som er end vcerre og mere umageligt , og efterfplge en <b>vis</b> fornuftig Javanesers <b>Lcerdom</b> , der trpstede sin Medborger paa saadan Maade . En Keiser udi Japan , saasom han var fodt under det Tegn af Hunden , fattede han saadan Mrbodighed for alle Hunde , saa at , naar en Hund dyde , befoel han , at Eieren deraf skulde felv vcere den til Graven . En rkiloLoMik mvdte En , der gik udi saadan Forretning ,

<h4>2382</h4>ved en Mode , hvor besvcerlig den end er , naar de indbilde sig , at den zirer dem , vilde HMgen besvcerge sig over saadant Forbud , og mange Mcend ere nu komne udi de Tanker , at deres Hustruers og Dsttres LEre beskyttes ved disse forstandsede Skiprter , der ere med Contrescarper og lern-Udenvcerker fortificerede , saa at den derfore ikke er saa let at bestorme som i gamle Dage . De tilskrive ogsaa hertil Aarsagen , at Kydflhed nu er stvrre end tilforn , og anftre til Beviis som <b>vise</b> , atTamper- Rets Sager meer og meer formindstes . Dette vil jeg dog lade staae ved sit Vcerd ; jeg vil allene sige , at mange Mcend , endogsaa de , som i Begyndelsen meest declamerede mod de harnistede Skiprter , nu holde dem for <b>Fruentimmerets</b> Zirat . Og sandeligen , naar man ret vil tale , saa maa man sige , at alt hvad som er bleven en fornemme Mode , er smukt og ziirligt , hvorudover jeg gierne vil tilstaae, at det er en Skrøbelighed hos mig , at jeg herudi ikke fplger Strommen . Det er derfore bedst at lade disse Skivrter blive urvrte , indtil en anden nyere Mode pdelcegger dem , hvilket maastee inden kort Tid vil hcende sig . Det var ikknn at vnste , at man ved Forandring ikke vilde henfalde til anden Extremitet , fom gemeenligen steer faavel udi Klcededragt fom udi Levemaader og Meninger ; saa at vore <b>Fruentimmer</b> , naar de forst engang finge Afsmag udi de harnistede Skiprter , ikke ville reformere dem faa stcerkt , at de blive forvandlede til tynde Kniplings-Klokker . Erfarenhed viser , at al Reformation fvrer stige Extremiteter med fig saavel udi Klcededragt , som Levemaade , samt <b>Lcerdom</b> og Meninger . Jeg siger ogsaa udi <b>Lcerdom</b> og Meninger. Den Engelske Skribent Blackmore haver viset , at

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016051048001'target=_>Westermeier, F.A.B., 1861, Christne Kirkes Historie</a><h4>409</h4>Næsten skrcekkeligere end nogensinde forhen bleve Forfslgelserne under Keiser Marcus Aurelius . Dette var saameget mere paafaldende, som denne Keiser stod i Ry for en ligesaa stor Fromhed og Dyd , som begge hans Forgjcengere . Den tidlige Indflydelse af en from <b>Moder</b> , til hvem hans barnlige Gemyt ganske hengav sig , havde han at takke for en oprigtig og hjærtelig Mrefrygt for det Guddommelige . Naar han opkaster det Spsrgsmaal „ Hvor har du seet Guderne , eller hvoraf erkjender du deres Tilværelse , siden du saaledes cerer dem ? , " saa svarer han sig selv : „ Af de Virkninger, som jeg stedse erfarer af deres Magt , erkjender jeg , at de ere til , og derfor cerer jeg dem saa inderlig . " Det var udentvivl Erfaringer af den eneste usynlige , men ham übekjendte Guds Kraft og Naade , hvilke han her beskriver . Og disse Erfaringer , hvilke han gjorde i sin Omgang med ham , ledede ham ogsaa til en <b>vis</b> Selverkendelse . Han bekjender : „ Forsaavidt som det afhcenger af Gudernes Bistand og Indgivelser kunde jeg allerede have opnaaet et med Naturen overeensstemmende Liv ; men naar jeg endnu ikke har opnaaet dette <b>Maal</b> , saa er det min Skyld , det ligger deri , at jeg ikke har været Gudernes Paamindelser og udtrykkelige Belærelser lydig . " Havde Keiseren vedblevet at bygge paa denne gode Grund , saa vilde visselig Fslelsen af denne Skyld have ført ham i Hans Arme , som alene er istand til at udslette denne Skyld ; men i Keiserens Indre fandtes endnu Meget , som var ham til Hinder heri . Han var paa den ene Side en Ven af den stoiske Philosophi ; og efter dens Grundsætninger trsstede han sig i Fslelsen af sin Skyld dermed , at ogsaa Menneskernes Forseelser vare nsdvendige til den hele Verdensplans Udvikling ; og hans naturlige Fromhed ledede ham meget mere til at cere sine Guder desmere og paa enhver Maade at opretholde Afgudstjenesten . Saaledes blev han ved det , som skulde have kunnet fore ham til den

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014032824001'target=_>Thorsøe, Alexander, 1879, danske Stats Historie fra 1814-1848, eller Fra freden i Kiel til kong Kristian den Ottendes død</a><h4>451</h4>Prins Kristian Frederik , celdste Son af Arveprins Frederik og Sofia Frederikke af Meklcnburg-Schwerin , var fodt paa Kristiansborg Slot den 18 September 17 L6 . Hans Fader var kun ndrnstet med eu jcevn <b>Begavelse</b> , hvorimod Moderen var en <b>Kvinde</b> med betydelige «åndelige Evner , der nedarvedes paa Sonnen . Den lille Prins fik en god Opdragelse , idet Tilsynet med hans Undervisning efter Ove Hogh Guldbergs Raad overdroges den klassisk danuede Sproglcerde Professor Niels Iversen Schow . I Skoleplanen var der i Overensstemmelse med cu Hovcdretuiug i Tidens <b>Aandsliv</b> lagt scerdeles Voegt paa Naturvidenskaberne, og Prinsen fattede nnder den kyndige Vejledning af Zoologen Hans Severin Holten og Mathematikeren Karl Ferdinand Degen en levende Intercsfe for disse . Desuden havde den retskyndige Kristian Kolbjornsen det Hverv at gjore Prinsen fortrolig med Lovgivningens Grnndscetningcr . De ncevnte Lcereres Undervisning var imidlertid ikke den eneste Vej til <b>Dannelse</b> for den unge Prins . I hans Faders huslige Kreds , paa Amalienborg og Sorgenfri , samledes den Gang en Mcengde af Notabiliteterne i Aandens Verden , han fik her jcevnlig Leilighed til at underholde sig baade med Videnskabsmand og Kunstnere. Prinsens ypperlige Natnrgaver udviklede sig under Vexelvirkningen med disse Mcend hurtig , hvorom hans i psykologisk Henseende saa meerkvcerdige Dagbogsoptegnelser afgive et Vidnesbyrd . " ) Da han var sytten Aar gammel , konfirmeredes han tilligemed sine to Sostre . Inlicme Sofia og Lonise Charlotte , af Viskop Balle i Frederiksberg Kirke . Kronprins Frederik havde vel i Overensstemmelse med sine

<h4>454</h4>militaire Interesser helst set , at den unge Prins havde helliget sig Militærtjenesten : men Kristian Frederiks Tilbøjeligheder gik afgjort i en ganske anden Retning , stont han steg til det hoieste Trin i det militaire Hierarki og forte Kommandoen over det Infanteriregiment , for hvilket hans Fader havde veerct Chef . Han fortsatte sine Sysler med Naturvidenskaberne og traadtc i Brevvcxling med disses ypperste Repræsentanter i Udlandet , men ved Siden deraf yndede han ogsaa den eedle Nydelse , som et , om end flygtigt , Studium af Kunstværker , kan forskaffe et for det Skonne modtageligt Sind . Dette var den theoretistc Side af Kristian Frederiks Liv ; men det var langtfra , at han af den forhindredes fra at nyde Selfkabslivets Adspredelser i fuldt <b>Maal</b> . I Hovedstadens glimrende Privatkrcdsc , i Statsminister E . H . Schimmelmanns og Gehcimckonfercnsraad Konstantin Brnns Hufc , var Prins Kristian Frederik en feteret Gcest . Her traf han fammcn med Tanmark- Norgcs <b>Intelligens</b> , med Alt , hvad der havde hullet fig til Samfundets Tinder , enten ved <b>Aand</b> , ved fornem Byrd eller ved stor materiel Formue . I disfe Cirkler , der bevægede fig om cesthctifl dannede <b>Damer</b> , og i den aristokratiske Klnb „ Harmonien " var Prinsen i sit rette Element. Vi spore den overordcnlige Lykke , han gjorde , i de symbolske Benævnelser , der fra den grcestc og nordiske Mythologi ovcrfortes paa ham . Der taltes om det „ Apollohoved " , han bar paa fine Skuldre ; han erklcercdcs for at vcerc „ stjon som Baldnr og veltalende som Vrage " . Under denne fine Omgang blev hans Vnscn tilslebcn , han tilegnede sig her den elegante Holdning , der iscer benndredes , naar han ved Jagterne viste stg til Hest , og han leertc at udtrykte sig med Lethed og Smag saavel i Modcrsmaalet som i fremmede Sprog . Men det er vel neeppe Tvivl underkastet , at al denne Glimmer , bag hvilke : : Livets sande Alvor ikke skjulte sig , havde en alt Andet end gavnlig Virkning paa Prinsen ; thi i den blcvc de rige Naturlag , der knnde have gjort ham til en stor Mand , snarere forflygtigede end virksomt udviklede . I Aaret 1806 kaldtes Priuseu bort sra Hovedstadens Selskabsliv for med sit Regiment at deltage i Beskyttelsen af Holstens Grumse nnder den fjerde Koalitionskrig . Samme Aar cegtcdc han i Ludwigslust Priuscsse Charlotte Frederikke as Mecklcnbnrg-Schwerin , der blev <b>Moder</b> til Prins Frederik , den senere Kong Frederik den Syvende . Dette Wgtestab var nlytteligt og hcevedcs 1809. Hidtil havde Prins Kristian Frederik ,

<h4>900</h4>* ) Grundtvig angiver i sit „ Kirke - Speil eller Udsigt over den kristne Menigheds LevnetslM " Kbh . 1871. 384 ff . sit Standpunkt saaledes : Jeg var nemlig faldet i dybe Tanker over den fortvivlede Stilling , hvori Christi Menighed og navnlig alle dens Bo " rn og ulcerde Medlemmer var kommet derved , at ncesten alle de Skriftkloge trodsig paastod , at ikke alene den Hellige Skrifts Oprindelse , Omfang , Mgthed og rette Fortolkning vare meget tvivlsomme , men at de kirkelige Grund-Lcerdomme om Treenigheden , Christi Guddom , og Forsoningen , naar man var Grund-Sprogene mcegtig og sllae ret til slet ikke fandtes i Bibelen , som hos alle Protestanter jo dog var Troens eneste Hjemmel . Hvor overbeviist jeg nemlig end selv var , om at Rationalisternes Bibel-Critik og Skrift-Fortolkning var omtrent lige ugrundede ug ucerlige , faa kunde jeg dog hverken oversee , at christelige Born og Lcegmcend var lige ilde farne , eller glemme , at selv min Bibel-Kundskab vilde ilke sorstaae til at trodse Legioners Vidnesbyrd , naar jeg ikke tillige havde kirkehistorisk <b>Kundskab</b> og havde fremfor alt et Vandens Vidnesbyrd i mit Indre , som kunde trodse al Verdens . Ligesom det nu blev mig soleklart , at han , som vidnede , at Saliggørelsens Hemmelighed var skjult for de Lcerde og Kloge og aabenbaret de Umyndige , kunde umulig uden klar Selv-Modsigelse binde de Ulcerdes Tro til de Lcerdes Vidnesbyrd , saa blev det mig ligesall soleklart , at hvis Skriften var den christnc Menigheds Troes-Regel , da vilde Vantroen nu , da ncesten alle de Skriftkloge hyldede den , have et langt gyldigere og swrkere Vidnesbyrd end Troen , som lun nogle enkelte Skriftkloge bevidnede , og min Slutning blev da , at saavist som Jesus Christus var Gud-Faders eenbaarne SM , saavist maatte der ogsaa i Kirken findes et langt gyldigere og startere Vidnesbyrd om den cegte , oprindelige , Christne Tro , end Bogstav-Skriften vaa nogen Mande kunde vcere for <b>Kvinder</b> og Bjsrn og alle de Ulcerde . Da jeg nu i denne Retning uafladelig grublede , lceste og strev under Bom og Paakaldelse , see ! da slog det mig i et velsignet Oieblik , at det magelose Vidnesbyrd , jeg saa mMommelig ledte om i hele AandenZ Verden , det som en Himmel-Rost hele Tiden og Christenheden i den apostoliske Troes-Bekjendelse ved Daaben . Hermed var for mig Lyset tcendt og Lodden kastet , thi at „ Vand-Badet i Ordet " altsaa Daabs- Ordet er den christelige Livs-Kilde , det havde Luther jo strax lcert mig , og at han , som selv havde indstiftet Daaben til et Gienftdelsens og Fornyelsens Bad i den Helligaand , ogsaa selv havde bestemt , vaa hvilken Tro der maatte dMes , naar man ved „ Tro og Daab " vilde vcere sin Salighed <b>vis</b> , det saae jeg strax , er saa « åbenbart uncegteligt , at hvem der ikke vil lade Troes-Ordet ved Daaben gioelde som et Ord til os af Herrens egen Mund , mcm enten forncegte Herren som Daabens Indstifter , eller forncegte Daaben fom Livs- Kilden , og derved stille sig selv udenfor den christne Menighed . Fra dette

<h4>2340</h4>Efter en Tale af Professor Ponlsen for det danske Sprogs Haandheevelse og Statens Opretholdelse , og efter at der fra „ Syvstjernen " var oplcest cn Skrivelse til den nordslesvigske Forening , fik Karl Plong Ordet . Han udviklede , at Nordslesvigerne i deres nationale Kamp ganske vist stnlde stole paa dem selv som paa Danmark , men det danske Folk var kun lille overfor det store Tydstland ; det var vel cn redelig og trofast Ven , men ingen stcert og mcegtig Forbundsfælle . Han udtalte sig derefter saaledcs : „ Og dog , slesvigske Bonder ! stnlle I ikke forsage , men frejdigcn udholde Kampen , thi I have hint uryggelige Rygstod , hin uovervindelige Forbnndsfcelle , som I soge og bchove , og Sejren er Eder forjcettct ! Det danske Folk staar ikke isoleret i Verden , det er ikke det sidste svage Skud af en alt udgaact Stamme ; men Stammen gror endnu frist og har stcerkc , lovrige Greue ; det har udcu for Daumark Brodre , som det elsker , og som elske det igen ; Brodrc , ikke blot i Herkomst og Sprog , men ogsaa i <b>aandclig</b> <b>Dannelse</b> og politisk Udvikling , i Sindelag og Strcebcn . Og disse Vrodrc bo ikke langt borte bag store Have eller fjerne Bjerge , saa at de kun se og hore lidt til os , saa at de ikke kende og ikke bryde sig om vore Anliggender ; men den faste Klippegrund , hvorpaa de bygge , udstrcekker bestormende sine Arme paa begge Sider af vore lave havomflydte Ocr ; ikknn cn Indso og ct Snnd cr imcllcm os , men de adstille os ikke , de forbinde os , de gynge os i hinandens Favn . I , danske Slesvigere, udgjore jo eet Folk med os , altfaa ere vore Brodrc ogsaa Eders . Vor fMes store <b>Moder</b> anerkender ogsaa Eder for sine cegtcfodte Born ; ville nn I fornegtc Eders <b>Moder</b> ? Det kundc heller ikke nytte Eder . hvis I gjorde det . thi Eders <b>Maal</b> rober Eder . I cr Skandinaver ! Ikke Slesvig , ikke Danmark , men hclc det store stonne Norden er Eders rette Fædreland ! Vel udgjor dette Fcedrcland kun i Tankens Verden en Enhed , vcl har Historien delt det i trende Stater med tvende Konger , og de Forsog , dcr hidtil have vceret gjorte paa at sorcnc det , ere mislykkede . Men genncmtrcengcr den falles Nationalbevidsthed

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012041108158'target=_>Fausbøll, V., 1866, Bidrag til en Ordbog over Gadesproget og saakaldt Daglig Tale</a><h4>9115</h4>Vor Glædes Kilde ! Dig vi svigter ikke ; Pocalen fyldes skal til bredfuldi <b>Maal</b> , Og mangt et Minde staaer for vore Blikke , Mens ud vi stikke , <b>Moder</b> , fro Din Skaal . Vor Moders Skaal . Den 16. Juli 1840. ( <b>Vise</b> i

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016041548026'target=_>Hansen, Maurits, 1857, Mauritz Hansens Noveller og Fortællinger</a><h4>2042</h4>„ Imod den udtccrede Fremmede scctter jeg min Manddomskraft , imod hans dybe <b>Visdom</b> , der dog ikle kan fattes af en <b>Kvinde</b> , scctter jeg mit Vid , mine Talenter , imod hans tvetydigc borgerlige Forholde mine Udsigter til en Embedsbane i Landets Skjod , min Onkels Penge , mine Evner til snart at fremtræde som fcedrenelandsi Digter . — O , Johanna ! Hvor er Du forblindct ! Men hvad hjælper det , at det lun er Forblindelse , nåar denne ikte kan forsvinde for Sandhedens Dag . — Mod , Kraft ! Jeg er Mand og jeg vil handle som Mand . I Morgen vil jeg gaa hen til Siewers . Med Alvor og Sindighed vil jeg opfurdre ham til en oprigtig Erklæring . Han sial , han vil give den . Kan han ikke godtgjore sine Hensigtcr for mig , saa vil jeg foreholde ham , hvad der er hans Pligt . Kan han det end , saa byder jeg ham en fredelig Kamp om det yndige Tvistens Wble . Han sial lære at agte , han sial beundre mig . Det er jo Mcrnd af <b>Aand</b> og Kraft , der værdiges at optages i den store , hemmelighedsfulde Forening . Velan , nu fan jeg give ham Bevis for , at hans ' Valg ikke var faldet paa en uværdig . Men hvad vil han med den jammerlige , raa , enfoldige Vonde ? Dog , ogsaa for denne er der vel Vrug iet saadant Eelsiab . Enhver vil faa den Plads , ham tilkommer " . Da den uindviede Adept længe nok havde uugget sig i Drsmme om sin tilkommende Virksomhed og Vetydenhed, fordrede Hjertet atter sine Rettigheder . Han bestuttede at udsse sine Fslelser i en Slegi , hvormed han næste Dag ester fuldbragt Veseg hos Agenten vilde lcegge sit Hjerte for den Skjgnnes Fsdder . Skade , at Digterens rsrende Poem er forkommet i Tidens Volger . Vi kunne ikke supplere dette Tab og lade os nsje med at forsikre , at det , deklameret med Pathos af Forfatteren , tog sig fortrcrffeligt ud .

<h4>2990</h4>emme Fslelser . Hvorledes det virkelig hang sammen med denne Gjengjeldelse, ville vi strår med et Par Ord forklare , men kunne ilke undlade, selv om en eller anden smuk <b>Lceserinde</b> ledes derved , at gjere den gammeldags Bemerlning : et Kjelken er en <b>Piges</b> rette Karakteerspejl . Fm Kobbertejet paa Veggene og de hvcelvede Fade paa Benken , fra hendes Forklcrde og Slcrven i hendes Hejre , fra Tyendets Virksomhed , Miner , Fortrolighed , fra hendes Ansigts Udtryk , hendes Stemmes Klang — erholder den ubemerkede Iagttager et Resultat for sit Studium af <b>Pigens</b> sande Vessaffenhed , hvad Hjerte , Sjcrl og Legeme angaar , imod hvilket hundrede Baller , Komedier , Spadsereture og literccre Samtaler ilke forståa . Men man vil indvende , at en <b>Pige</b> eller <b>Kone</b> skal ikke residere bestandig i sit Kjelken ; man vil ilke elske og egte en Husholdersie; man ensier en fin , dannet , em , hejt felende Ledsagerinde igjennem Livet . Meget vel ! Men alt dette legger hun for Dagen i sit Kjelken og mere til . — Ieg ser gjerne en tcekkelig <b>Pige</b> ved hendes Vrodeer-Ramme , glcrder mig ved hendes Klaveerspil , hendes Sang , sinder usigelig Behag t at fere en let , overftadisi Samtale med hende om hendes Unglingsdigter Schiller , Schoppenhauer , W . Scott . Men jeg paastaar dog , der mangler hende det vesentlige , sandt <b>kvindelige</b> Tekke , hvis hun ikle skylder sit Kjellen og sine Smaasessendes Kammer en Del , en stor Del as sin <b>Dannelse</b> . Men holdt ! Nu er jeg igjen hos Letchen , der opremt leder det virlsomme Kjellen » ersonale . Vi ville ille lengere opholde os ved Besirivelsen af den virkelig yndige <b>Piges</b> Udvortes; lun et Par Linjer til hendes Karalteristil . En <b>vis</b> regelmessig Fasthed , en <b>vis</b> scedelig Orden var fra Barndommen af fremtrædende hos Pigen . Denne Egensiab gav hende et Herredemme over sig selv og over alle sine Omgivelser , som hevede hende i alles Agtelse uden at falde nogen besuerlig ; thi det var saare langt fra Letchen at ville herske . Hendes Forstand var lys , og hun besad nogen Vittighed , hvillen hun iscer evede imod de mange Tilbedere , der sra tidligt havde svcermet om den smukke , stinke , kloge og — som man i Almindelighed troede — rige Jomfru . Imidlertid siulde vi aldeles tåge fejl , dersom vi tenkte os Lettina Vieder som en kold Pallas Athene . ANerede i hendes tiende Aar skrev den beremte Fysiognomist fra Zurich , da han engang lenge

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011032520038'target=_>Collett, Camilla, 1855, Amtmandens Døttre</a><h4>1170</h4>Et Forhold mellem to unge Pcrsoner , der knn grundct paa Interesse ikke har et bestemt andet <b>Maal</b> end en blot <b>aandclig</b> Udvezling , horer af gode Grunde til de saare sjeldne hos os . De synes at tilhore lykkeligcre, fricrc , mere udvikledc Tilstande end de ncrrvcrrcndc. Naar de dcsuagtet finde Sted , vcrkke de en Opsigt som alle Phcrnomener , der ikke spire i en naturlig Grund . <b>Modre</b> og kunne ikke fordrage den Slags Venstaber . De cre de-m for upraktiske , ofte hindrende for Dsttrencs Lykke . Og Modrene have Ret . Hvortil denne farlige Prsvesteen ? vore Born kunne blive lykkelige ad den almindclige Vci , uden at der spsrgcs om de have nogen <b>Aand</b> eller <b>Sjel</b> . . .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013012424002'target=_>Holberg, Ludvig, 1858, Hundrede og tyve af Holbergs Epistler</a><h4>655</h4>og medens jeg klceder mig , bander min Skrceder : c . Det sidste skeer , efterdi mine Klceder ere saa trange , faa at det er fast det samme som at fyre sig i Harnist . Mig synes , at Verden er trang nok , saa at man ikke haver ny digt at gipre den trangere . Du vil uden Tvivl hertil svare : hvad kan Skrcederen givre dertil , at Klcederne blive saa umagelige og piinagtige ? Han gipr jo ikke andet end hvad Folk vil have . Intet er vissere , hvorudover , naar jeg tcenker mig ret om , saa haver jeg heller Aarsag til at vcere vred vaa mig selv , der efter Moden bestiller Klcederne , end vaa Skrcederen , der giyr dem , eller rettere vaa Moderne , der tyrannisere baade over Skrcederen , over mig og over alle andre , og det udi saadan Grad , at det holdes for en Egensindighed , ja hartad en Rebellion at scette sig derimod , hvorudover man maa med Taalmodighed stikke sig derudi , som udi andre Ulejligheder, hvilke man ei kan hindre , og sige med Bonden hos ? 6 ti ' onium , der havde mistet sin brogede So : mo6o » ie , moclo Bic . Vi maae takke Gud , at de Parisiske Skrcedere, som fabriqvere vore Klcede - <b>Moder</b> , ikke foreskrive os det , som er end vcerre og mere umageligt , og efterfplge en <b>vis</b> fornuftig Javanesers <b>Lcerdom</b> , der trvstede sin Medborger vaa saadan Maade . En Keiser udi Japan , saasom han var fvdt under det Tegn af Hunden , fattede han saadan JErbpdighed for alle Hunde , faa at , naar en Hund dpdc , befoel han , at Eieren deraf stulde selv bcere den til Graven . En rkilosopKuß mvdte En , der gik udi saadan Forretning , og , da den samme besvcergede sig over sligt hacmligt Hoveri , trvstede han ham med disse Ord : vi maae takke Gnderne , at Hans Keiserlige Majestcet ikke er fvdt under det Tegn af Hesten , saa havde vore Vilkaar herudi end bleven slettere . Orientalske Folk , hvis <b>Moder</b> og Levemaader vi udi mange Ting belee , fortiene derudi at berammes , at de stedfe beholde den samme Klcededragt . Europceere derimod fsre sig op som

<h4>1151</h4>ved en Mode , hvor besvcerlig den end er , naar de indbilde sig , at den zirer dem , vilde HMgen besvcerge sig over saadant Forbud , og mange Mcend ere nu komne udi de Tanker , at deres Hustruers og Dpttres LEre beskyttes ved disse forflandsede der ere med Contrescarper og Jern-Udenværker fortificerede , saa at den derfore ikke er saa let at bestorme som i gamle Dage . De tilskrive ogsaa hertil Aarsagen , at Kydsthed nu er stvrre end tilforn , og anfore til som <b>vise</b> , atTamfter- Rets Sager meer og meer formindskes . Dette vil jeg dog lade staae ved sit Vcerd ; jeg vil allene sige , at mange Mcend , endogsaa de , som i Begyndelsen meest declamerede mod de harniskede Skiprter , nn holde dem for <b>Fruentimmerets</b> Zirat . Og sandeligen , naar man ret vil tale , saa maa man sige , at alt hvad som er bleven en fornemme Mode , er smukt og ziirligt , hvorudover jeg gierne vil tilstaae, at det er en Skrøbelighed hos mig , at jeg herudi ikke spiger Strpmmen . Det er derfore bedst at lade disse Skivrter blive uwrte , indtil en anden nyere Mode pdelcegger dem , hvilket maastee inden kort Tid vil hcende sig . Det var ikkun at pnste , at man ved Forandring ikke vilde henfalde til anden Ertremitet , som gemeenligen skeer saavel udi Klcededragt som udi Levemaader og Meninger ; saa at vore <b>Fruentimmer</b> , naar de forst engang finge Afsmag udi de harniskede Skiprter , ikke ville reformere dem saa stcerkt , at de blive forvandlede til tynde Kniplings-Klokker . Erfarenhed viser , at al Reformation fprer stige Extremiteter med sig saavel udi Klcededragt , som Levemaade , samt <b>Lcerdom</b> og Meninger . Jeg siger ogsaa udi <b>Lcerdom</b> og Meninger. Den Engelske Skribent Blackmore haver viset , at den Engelske Nation udi hvert Seculo haver havt en ny Smag udi Skrifter , saa at , hvad den i een Alder haver admireret , haver den i en anden Alder holdet latterligt og

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012060806013'target=_>Bruun, Christopher, 1878, Folkelige Grundtanker</a><h4>1534</h4>vanter ? Jo , for at der es Døtre igjen kan oplæres til at blive Guvernanter , og blive duelige til at undervise i det , som hverken de seiv eller deres Lærlinger nogensinde faar Brug for . Det hele er det argeste Tant . I Virkeligheden gjør det ingen anden Tjeneste , end at fodre Forfængeliglieden i de unge Damer, nåar det kan siges om dem : hun kan tre Sprog . For at naa dette <b>Maal</b> er det , at Halvdelen af dem læser sig blegsottige og nervesvage , mens de gaar paa Skolen . Men bagom al denne Meningsløshed dølger der sjg lidt af en virkelig Mening , som rigtignok er grundfalsk , og det er den , at den bedste Undervisning man kan faa i de unge Aar , er Undervisning i fremmede Sprog . Og denne falske Mening har sit Udspring fra Latinskolen. Vore høiere Pigeskoler er ligefrem en Hunkjønsform af Latinskolen , dannet efter den latinske Grammatiks Regler . Og de sproglærde unge <b>Damer</b> , som gaar der , faar ikke mere af virkeligt Indhold for sin Sjæl , end deres classiske Brødre paa de lærde Skoler . Pigeskolen giver en ligesaa daarlig Forberedelse til det , at blive <b>Kvinde</b> i <b>Aand</b> og Sandhed , som Latinskolen giver til at blive Mand .

<h4>2851</h4>gyndt at arbeide for . Men den Maade , hvorpaa den raadende <b>Dannelse</b> <b>moder</b> dette Arbeide, vidner vel stærkere end noget andet om hvor blottet denne <b>Dannelse</b> er for <b>Aand</b> . Flere af dens Talsmænd optræder endog ligefrem med kaad Haan imod det ægte norske Sprog og Tanken om at løfte det op igjen til dets gamle Værdighed . Og de faar af sine Standsfæller Lov til at gjøre dette ustraffet .

<h4>3627</h4>for den høiere <b>Dannelse</b> . Og i dette G-rundlag er det , at <b>Kvinderne</b> ikke faar Del . Deres Undervisning faar søge sig sin G-rundvold andetsteds . Kan man saa undre sig over , at de er fremmede for Mændenes <b>Aandsliv</b> og deres Tanker . I Sandhed , hvis man ligefrem havde sat sig fore at fjerne Mand og <b>Kvinde</b> aandeligt fra hinanden , kunde man ikke let finde et virksommere Middel , end det , at bygge deres Skole paa hver sit G-rundlag .

<h4>3629</h4>En i Sandhed folkelig ( eller national ) <b>Dannelse</b> , en historisk-poetisk Skole , hvor Modersmaalet i alle Maader er Hovedsproget , er det som skal til for at udjevne Kløften . Den vil hjælpe Mændenes <b>Aandsliv</b> til ikke i indbildt Overlegenhed at vende <b>Kvinderne</b> Ryggen, men til at gaa dem i Møde , som Broder mødes med Søster . — Og det er den samme Slags Skole , som <b>Kvinderne</b> trænger til , om de skal faa det Høisind , som giver dem Lyst til at dele Mændenes <b>Aandsliv</b> . Den nuværende Pigeskole med sine » tre Sprogc , og al den øvrige Humbug opdrager ikke til Tanker , men til Pjat . De trænger til at Digternes Idealer faar udvikle deres Sjæls Vinger . De trænger til at se hvadMænd har arbeidet og kjæmpet forfør i Tiden , for levende åtkomme

<h4>3747</h4>Vi trænger nu høilig til Lægedom for denne Blegsot , som har grebet vort <b>Aandsliv</b> og afkræftet detlige ind i Margen . Vi trænger til at tænke vore Tanker om igjen og give dem Personlighedens Vægtfylde . Vi trænger til at tåge os sammen ud af denne spredte Mangfoldighed , ud af disse talløse Smaatanker , og samle os paa Hovedtankerne . Og dem trænger vi til at tænke igjennem til Bunds , saa de kan faa Betydning for vort Liv og føre os til Handling , saa de kan give os noget at arbeide for og leve for . — Men har vi faat vore vigtigste Tanker saaledes gjennemarbeidet , saa vil vi nok ogsaa vide at dem paa vor Moders <b>Maal</b> . Saa vil vi lære at meddele dem til vor Søster og til vor <b>Hustru</b> . Og kan vi det , saa er vi paa god Vei til at kunne gjøre dem forstaalige ogsaa for det menige Folk . De Tanker , som blir en virkelig Magt i vort eget Indre , de kan ogsaa blive en Magt ucfover . Men den Tanke , som skal blive en Magt i Verden , den maa gjennen.bryde det Hegn , som omgiver Studerkamrene og Bøgerne. Den maa kunne følge Bonden under hans Arbeide og Moderen mellem hendes Børn .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013022624015'target=_>Holberg, Ludvig, 1867, Ludvig Holbergs Kirke-Historie</a><h4>674</h4>det meere af Hyklerie end af et got Hierte , thi han beholdt selv endeel af det soldte Gods og overleverede det svrige « ) . Til ham sagdest . ? 61161 - : hvi har dn ladet dig friste af Satan til at lyve for den Hellig <b>Aand</b> ? Hvorpaa faldt straz- dod paa Jorden . Hans <b>Hustru</b> lod sig kort Tiid derefter indfinde , ey vidende , hvad som steed var ; Hende spurdte og Apostelen ligeledes til , og fik det famme Svar ; Hvorudover han sagde : I have begge to overlagt at friste GUds <b>Aand</b> ; see ! de , som have begravet din Husboud , skal ogsaa begrave dig , hvorpaa huu ogsaa faldt dsd for hans Fsdder ; Og foraarsagede dette Mirakel en Redsel hos alle dem af Meenigheden , fom det Horte . Men dette saavel som andre Paafslgende Mirakler foraarsagede , at den ypperste Prcest med sit Auhaug paa nye kastede Apostlerne i Fcengscl ; Men en Engel losede dem deraf , saa at da Raadet skikkede Bud efter dem , fandt man ingen , endskiont Fcengslet var tillakket . De vare da allercede ndi Templet , hvorndaf den ypperste Prcest lod dem fore for Raadet , og talede dem saalcdcs til : Vi have jo forbudet eder at lcere udi lEsu Navu , hvortil de svarede : At man maatte meere adlyde GUd end Menneskerne , og vidnede , at lEsus var Verdens Forloser . Hvorudover Isderue bleve saa ophidsede, at de vilde lade dem omkomme . Men en gammel Lcerer af den Pharisan ' ske ved Navn raadede dem derfra, sigende : Dersom deres <b>Lcerdom</b> kommer fra Menneskerne, vil den falde afsig selv , men hvis den kommer fra GUd , faa kand ingen ftaae derimod . Dette Raad blev efterlcved , og Apostlerne komme anden gang igicn paa fri Fod .

<h4>1035</h4>som er Hoved for den Boot kalden var fra og holdes for at have begyndt at udstrse sin falske <b>Lcerdom</b> udi det Aar 172. Han indtog straf mange af de Troende ved udvortes Dyder og en Meotel-el ! koeniteiMe , faa at han for en Mand , der var begaven med Nii-aclers og Provhetiers Naade ; Ja han kaldte sig selv Trssteren , den Hellig <b>Aand</b> , som af Christo var loved Asiostlerne . Til ham fsyede sig tvende <b>Qvinder</b> , nemlig ? i-iBcilw og Nnximiiln hvilke vare af hsy Slcegt og bemidlede , og trcekkede mange til sig , ikke allene ved deres Tale , men end og med Skienk og Gaver . De lcerte tilsammen , at GUd i Begyndelsen havde vildet frelfe Verden ved UoB6B og Provheterne , men forgieves , hvorudover han havde fkrcedet til Manddoms Paatagelfe , og , fom det ey heller nogen Virkning knnde have , var han ved den Hellig <b>Aand</b> nedstegen udi Nonwnus , ? i ' l8l ) illtl og Nsximillg . Dette foraarfagede , at Non- WnuB af sme Tilhceugere gemecnligen blev kalden Trsstercn . Disse Folk svede saadan stcerk v ! Bciplino . at de uddreve deres Tilhcengere af deres Samqvem for de allerringeste Forseelser . De bleve siden deelte ndi og l ' i-ocli . Nem vi-Boiule, og varede dette Kietterie en lang Tiid .

<h4>1872</h4>Adskillige begyndte da at separere sig fra Nestorio , anseende ham som en Kietter , og en Biskop ved Navn kroelus holdte paa en <b>vis</b> Fest-Dag en Prcediken , hvorudi han vidtlsftigen igiendrev Nestorii <b>Lcerdom</b> og beviisede , at det er rett at sige , at GUd har lidet og er dod for os , og at lomfrue Naria bor kaldes eller GUds <b>Moder</b> . Denne Prcediken jo meere den behagede Tilhorerne , jo meer ophidsede den saa at han udi 3 andre Prcedikmer strax derpaa strcebede at rekutere kroolum igien , dog uden at ncevne ham . Disse Prcedikmer bleve siden samlede udi en Bog , og allevegne omstillede , saa at der reisede sig Disputer derom , saa vel udi de Vestlige som Bstlige krovinoier , og mange bleve heel ivrige i denne nye <b>Lcerdom</b> . Blant Modstandere var den storste ( Drillus , Biskopen af xanck-ia , hvilken strar hvessede sin Pen mod ham , ja lod ham viide , at han ikke knnde vcere udi kommunion med ham , med mindre han stod fra sin <b>Lcerdom</b> .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111601019'target=_>Skaar, J.N., 1879, Norsk Salmehistorie</a><h4>5027</h4>oegte Hymne-Kvad  ambrosianske Salmer " . Den rette fyldige , klare , henrivende Tone var engang given ; Sporene kunde aldrig forlades , saalcmge Kirken bevarede sin Tro og sin <b>Aand</b> . Man sang sig md i disse mesterlige malmer , og sang sig derved md i Kirkens Bekjendelse, i dens verdensovervindende Tro , i dens aldrig beskjemmende Haab , i dens uforgjengelige Kjoerlighed " . Haand : Haand med Salmedigtningen gik Ambrosius ' s Bestrcebelse for at fremhjcelpe Kirkesangen , idet han tog den grcestmusikalske <b>Dannelse</b> til Hjcelp og skabte en livlig og dog regelbunden Salmesang , hvori den hele Memghed kunde tåge Del . Hvilket Indtryk denne Sang gjorde paa Augustin , derom have vi ovenfor ( under Nr . 15 ) lM hans Vidnesbyrd :  Det var ei for lang Tid siden — siger han ved samme Anledning — at den mailandske Kirke havde begyndt at feire denne Slags TrM og Opmuntring , hvorved alle Brpore med stor Lyft , med Hjerter og Raster , sang sammen ; det var netop dengang , da det kristelige Folk holdt Nattevagter i Kirken , beredt paa at dF med sin Biskop , din ( Guds > Tjener , og da min <b>Moder</b> ( Monika ) , din <b>Tjenerinde</b> , fremmest blandt de aarvagne . ganske levede i sine BMner . Vi , som dengang endnu ei var opvannede ved din Aands GliZd , blev dog vakte , da hele Staden var i Uro og Forsceroelse . Da var det , at den Indretning kom i Gang , at Hymner og Salmer blev afsungne efter O rien : ens <b>Vis</b> , paa det Folket ei skulde forfmcegte as Sorgens Lede ; og fra den Stund af er det nu saaledes bibeholdt til den Dag idag , da mange , ja fast alle dine Menigheder i de andre Lande efterlignede det " . Ogsaa Am bro si us selv udtaler sig om den mcegtige Virkning , den af ham indfMte Salmesang Mede .  Man raaber paa " , siger han ,  at Folket skal vcere forlokket ved min Hymne-Sang . la , det vil jeg heller ikke fragaa . Thi unegtelig , ligger der noget stort oeri , det n : cegtigste af alt. Thi hvad kan vel vcere magtfuldere end Treenighedens Bekjendelse , som daglig udstrMmer fra hele Folkets Lceber . Alle kappes om at bekjende Troen ; de have lcert i Kvad at prcedike Faderen og Sønnen og den Helligaand . Saaledes ere alle blevne Lcerere , som ellers , neppe kunde gjcelde for Disciple " ^ ) . Maaste var  Veni resswptni- Fyutiuin " netop en af de Hymner , som Menigheden istemte under sin Nattevagt i den mailandske Kirke i Aaret 386. Medens Arianerne fornegtede Kristi Guddom , have vi i denne Salme en Bekjendelse af Kristus som 31323 FSiuinN LudstantiN ,  en sand Gud , fpdt af Faderen fra Evighed , et sandt Menmfle , ftdt af Jomfru Maria " .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006082800008'target=_>Sundt, Eilert, 1855, Om Giftermaal i Norge</a><h4>8117</h4>62. Jeg tanker mig et Land , hvor et Aarhundrede igjennem Tilstanden har varet nogct nar uforandret , hvor Folkemcrngden ikke er voxet , hvor Industrien og ^ Virksomheden og Oplysningen ikke har udviklet sig , hvor saa at sige Tankerne have staaet stille og den ene Slagt i magelig Ro har indskranket sig til at overlevere i uforandret Stand den fra Fortiden modtagne Arv af Eiendele ag Begreber og Sader til den naste ; jeg tcrnker . mig derncrst , at verdenshistoriste Vegivenheder ftludselig medfore en Forandring i hine tilvante Forholde , og det en saadan velgjsrende Forandring , at Menneske-Aanden rsrer sig friere , at Livet udvikler sig livligere , at Tanksomheden og Friheds-Sindet og Foretagelses-Aanden og Wirke-Lysten ytrer sig kraftigere , at flere Naringsveie aabnes , sterre Forraad samles , flere Huse bygges og nye Familier dannes og flere Bern voxe op , og Fordringerne til Livet blive stsrre , og Menneskene sattte sig alt hsiere og hoiere <b>Maal</b> for sin S traven , og Folkemcrngdens Tilvart tvinger Slcrgtcrne til altid ivrigere at anstrcrnge sig for at holde sig oppe paa den allerede naaede Hside . — Dette sidste Afsnit af Folkets Liv vil da <b>vise</b> sig som en Civilisationens Tid mod de foregaaende Tiders levnhed og Enfold . Men hvad Indstydelse vil den nye Tingenes Orden have paa Todeligheden i Laudet ? Rimeligvis en dobbelt . Te Mcrnd og <b>Kvinder</b> , som ere i den Alder , at de maa arbeide for sig og Sine , og som rives

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081403021'target=_>, 1863, For Hjemmet</a><h4>2133</h4>ders eneste Son , den Marston , som allerede er prcesentclet for Lcescren . Onkelen havde bestemt den unge Mand for Kirken , idet hau haabcde , at han engang skulde blive hans Eftermand i det Prsstekald , som han havde og som i lange Tider havde vcrret i Familien . Men Maistons <b>Begavelse</b> og Lyst gjorde ham langt mere stikket for Skranken . Han var i Besiddelse af ypperlige Evner , og Dr. Howard havde fsr sin Dsd den Glcrde at see ham i en uafhcrngig og anseet Stilling paa den Bane . han havde valgt . Mrs . Howard havde stedsc deelt fin Mands Fslelser for Ndoptivssmien . Hun holdt inderlig af ham og efter Doctorcns Dod vcdblcv hendcs Huus at vcere den unge , Mands Hjem . Hun var en god , elskvcerdig <b>Dame</b> . der ingenlunde manglede <b>Begavelse</b> . Hendes Liv havde voerct tilbragt vcesentlig blandt Mcrnd af <b>Dannelse</b> og <b>Lcerdom</b> , og , skjent hun selv hverken var videnskabcligen eller kunstnerist dannet , besad hun i en ikle ringe Grad den Sands og den Evne til < at bedsmme , som M < rnd af Talent scrtte Priis paa . Hcndes svagcstc Side var altfor store Tanker om det stcrrkere Kjsn . Denne svage Side var , saalcdes som ofte er Tilfwldct , kun en god Side , drevet til Uderlighed . Den hidrsrte fra hcndes dybe Kjcerlighcd og Veneration for hendes egen Herre og Mester . Hun tillagde ham alle Aandens og Hjertets Fortrin . og hun stuttcde kjcrrligen fra ham til alle andre og kom ad en dyb Ovcrbeviisnings Vei , ikke paa Grund af nogen Mangel paa Styrke i Charakteren , til at see op til Mandene .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016101948007'target=_>Merle d'Aubigné, J.H., 1877, Luthers Ungdom</a><h4>1471</h4>Berthas Tante var ikke hvad Verden kalder en gudlss og ond <b>Kvinde</b> , tvcrtimod iagttog hun alle Christendommens udvortes Skikke , gik i Kirke , lceste og bad ; men Kjcrrlighed til Rigdom , denne Rod til alt ondt , havde ganske fcrngslet hendes Sjcrl til Jorden . Hendes <b>Aand</b> sad ligesom indestængt ien liden trang Skal , som blev mere trang for hver Dag , indtil den truede med at kvcrle den sidste Livsgnist . Lige fra sin Barndom havde denne stakkels <b>Kvinde</b> ikke hcwt noget høiere <b>Maal</b> end at samle de forgjcrngelige Skatte , som Mol og Rust kunne fortcrre . Maatte hver og en af vore unge Lcrscre vaage og bede imod en Synd , som saa let udsletter alle bedre Indtryk og lamstaar alle crd- - lere Krcrfter .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006081800017'target=_>Bruun, Christopher, 1878, Folkelige Grundtanker</a><h4>658</h4>Men denne « huslige Lyksalighed » er ogsaa den eneste høiere Livsmagt som vor <b>Dannelse</b> for Alvor har lært at kjende . Ja , alt for ofte er det netop Hen som slnger og tilintetgjør alle He andre . Hvad Manden ellers kan have lært at elske og sværme for i sine Ungdoms Dage , foruden den <b>Kvinde</b> han har faat kjær , det faar han være rede til at opgive som en fager , men uholdbar Drøm , naar « Livets Alvor-» melder sig i Skikkelse 3. t <b>Kone</b> og Børa . Mandens Livsgjerning, selv Embedsmandens , har for det meste sin høieste Betydning i det at skabe disse et lykkeligt Hjem . Det er som om vore dannede selv halvveis skulde vide af , at de ikke har faat nogen høiere Livsopgave end denne : at opdrage en Efterslegt som er bedre end de selv . Hvor mange Nord- MNnH 2 uHeB der vel , som har sat sig et aandigt <b>Formaal</b> for sit Liv , og som med G . A . Krog kan <b>vise</b> det den samme Troskab som NuuHreHer idlauHt os viser i Arbeidet for <b>Hustru</b> og Børn ?

<h4>891</h4>ham fast ide Egne hvor hun selv kan være med. Oss desuden kan det let hænde at en Mand som var en alvorlig Kjærlighed til aandelige <b>Formaal</b> , vil komme i en saadan Stilling, at denne Kjærlighed kræver alvorlige Ofre af udvortes Velvære . Og disse Ofre vil ramme hans <b>Hustru</b> mindst ligesaa føleligt som ham selv . Har hun nu ingen Sands for det som krgzvsr disse Ofre , 83,3 , er det da 33,3 . menneskeligt naturlisst at hun LuHsr det urimsiisst , om Nl3 , Msu for den Sags Skyld vil gjøre Skaar i hendes og Barnenes Lykke . Hun vil for det meste opdv « ls al sin Indflydelse for at faa Manden til at se at « i alle Fald i dette Tilfælde » kræves der ikke uoFst BaaH3 , ut Offer ak ham . Og en saadan Kamp med en elsket og elskende <b>Hustru</b> er ikke saa let . Mangen Mand som har Bt3 , at Bt33rk overfor alle Modstanderes Angreb , har bukket under for sin Hustrus Bønner og hendes Kjærlighed . Paa denne Maade kan Hustruens Ligegyldighed for Mandens <b>Aandsliv</b> let gaa over til at blive en ligefrem Hindring for det . Og det gjør den meget ofte . En af vore ypperste Videnskabsmænd og Universitetslærere har udtalt , at naar der var en ung Mau6 som der vaagnede et virkeligt videnskabeligt Liv hos , saa varte det for det meste ikke længere , end til han blev forlovet . Fra den Stund af var han tant kor Videnskaben . Og en af de faa ældre Mænd jeg har kjendt , som vidste hvad det var at arbeide og at ofre noget i en stor Sags Tjeneste , udtalte noget tilsvarende , kun endda stærkere . « Den som er forlovet , er død for Sagen . Der er to Slags Lyksalighed til i Livet : det er den huslige Lyksalighed, og den ideelle . Vi skal ikke vente at ét Menneske skal kunne eie dem begge . Den som har <b>Kone</b> og Børa , han kan bringe Offer for dem , men ikke for nogen anden Sag » .

<h4>907</h4>Overhoved maa det vel være greit at Latinskolen er en meget stsork Bllillemur mellem I > lanH og XvinHs . Latinen er jo ikke i denne Skole et enkelt Fag ved Siden af de andre . Den er Hovedfaget ; Tanken er at den ( og ( FlNBli6n ) skal være Grundlaget for den høiere <b>Dannelse</b> . Og i dette Gr un dl ag er det at <b>Kvinderne</b> ikke taar Del . Deres Undervisning taar ss ^ s sig sin ftrunHvolH auHetBteHB . Kan man saa undre sig over at de er fremmede for 2 l « kuHeneB <b>Aandsliv</b> og deres Tanker ? I Sandhed , hvis man likefrem navHe sat sig fore at fjerne Mand og <b>Kvinde</b> aandeligt fra liiuananden, kunde man ikke let finde et virksommere Middel , end det at bygge deres Skole paa hvert sit KruuHiag .

<h4>908</h4>En i Sandhed folkelig ( eller national ) <b>Dannelse</b> , en historisk-poetisk Skole , hvor Modersmaalet i alle Maader er Hovedsproget, er det som skal til for at udjevne Kløften . Den vil hjælpe Mændenes <b>Aandsliv</b> til ikke i indbildt Overlegenhed at vende <b>Kvinderne</b> Ryggen , men til ai , gaa Hem i Møde , som Broder mødes med Søster . — Og det er den samme Slags Skole som XvinHerne tlsen ^ er til , om de skal faa det Høisind

<h4>918</h4>Vi trænger nu høilig til Lægedom for denne Blegsot , som har fredet vort <b>Aandsliv</b> og afkræftet det lige md i Margen . Vi trænger til at tænke vore Tanker om igjen og give den Personlighedens Vegtfylde . Vi trænger til at tåge os sammen ud af denne spredte Mangfoldighed , ud af disse talløse Smaatanker , og samle os paa- Hovedtankerne . Og dem trænger vi til at tænke igjennem til Bunds , saa de kan faa Betydning for vort Liv og føre os til Handling , saa de kan give os noget at arbeide for og leve for . — Men har vi faat vore vigtigste Tanker saaledes gjennemarbeidet , saa vil vi nok ogsaa vide at udtrykke dem paa vor Moders <b>Maal</b> . Saa vil vi lære at meddele dem til vor Søster og til vor <b>Hustru</b> , og kan vi det , Baa er vi paa god Vei til at kunne gjøre dem forstaalige ogsaa for det menige Folk .

<h4>1191</h4>Men videre sagde den Mand som jeg nævnte , at de som blir stærkt grebne af Sandsen for det menneskelige , let blir saa optagne af det , at de glemmer Christendommen . Denne Indvending er slet ikke — saaledes 3 om den foregaaende — grundet paa Misforstaaelse . Jeg tvivler slet ikke paa at der kan findes dem at vore unge Tilhørere , som glemmer Gud over det menneskelige . Vi Mennesker er meget tilbøielige til at glemme Gud . Den ene glemmer ham over sin Gaard , den anden over sin Handel , den tredie over den <b>Hustru</b> han har kjær . Den fjerde , kan vi saa godt tilføie , over sin Videnskab , — og saa fremdeles . Men derfor skal vi ikke ønske bort hverken Videnskab eller Jordbrug eller Handel eller huslig Lykke . Hvor mange Mennesker inden den dannede Stand er det ikke , som glemmer det ene fornødne over sine dannede « Interesser » . Paa lignende <b>Vis</b> vil det selvsagt ogsaa gaa med den <b>Dannelse</b> Folkehøiskolen har at byde . Men det tænker jeg vel vi tør sige , at de som glemmer Gud over det menneskelig ædle og skjønne , de vilde ellers have glemt ham over andre lavere Ting . Er dette menneskelige virkelig af det gode , saa har det naturligvis ikke i sig selv nogen Tilbøielighed til at drage vort Sind bort fra Gud .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009072112002'target=_>Collett, Camilla, 1877, Fra de Stummes Lejr</a><h4>1121</h4>Det er dette af Christus selv indstiftede Forhold , der aldrig kan ophæves . Hvor det sker , synker Samfundene sammen — Historien kan fortælle det . Ingen af Parterne kunne existere uden i denne nære , inderlige Sammenhæng med hinanden . Hvor Manden vil løsrive sig i selvstiftet Frihed , drager han <b>Kvinden</b> med i Fordærvelsen ; vil han rettelig beholde hende , maa han bøie sig til hende . I ret <b>Aand</b> at befæste og opretholde dette Sambaand bør være <b>Maalet</b> for alle sande Emancipationsbestræbelser. Hvor dette <b>Maal</b> lades ude af Syne , ville de udarte som vandsyge Væxter .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015082448274'target=_>Landstad, M.B., 1871, Kirkesalmebog</a><h4>3927</h4>Du er alene den , som giver de Trcette Kraft og formerer den Vanmcegtiges Styrke , o saa giv ogsaa mig Svage Kraft og styrt mig Dag fra Dag herefter at blive from og leve bedre . O Gud , hold dit Ord og dine hellige Sakramenter vedlige hos os evindelig uden falsk <b>Lcerdom</b> og vrang Gudstjeneste, at jeg og andre bodfcerdige Syndere altid maa finde Kraft deri til Salighed og en Livslugt til Livet . O Herre , bevar Kongen og Landet med al kirkelig og verdslig Ovrigbed fra al Ulykte . Gud give os Fred og Rolighed . Gud unde os Helbred og Sundhed . Gud naadigen velsigne vor ( Ager og Eng ) Mad og Drikke , vor Kurv og vore Levnmger. O Herre , toeuk i Naade paa alle Syge og dem , som lide Angest og Pine . Giv dem en god Nat , send Lindring og Lcegedom , send dem Trost og Haab . Hjcelp dem Ul at kunne ' vare glade i Haabet . taalmodige i Trangselen og varagtige i Bsnnen , og lad deres Forlssningstime , som bie paa dig , ikle vare langt borte . O Herre , vcer mig arme Synder ( min Husbonde , <b>Hustru</b> , Foraldre , Born ) min Slcegt og Venner fjern og noer og alle Mennesker naadig for Jesu Kristi Skyld ! Vogt os med Hus og Hjem , Gods og Formue fra Ilds-Nod , Vands-Vaade , Tyvehaand og alt andet Ondt , og lad ingen Ulykke komme inden Dore . Der lagges Dage , Uger og Aar til min Alder . Vor Klokke staar . vor Tid bortgaar — og Evigheden forestaar . O Herre , lcer mig saa at talle mine ' Dage , at jeg maa bekomme <b>Visdom</b> i Hjertet ! Lcer mig saaledes at bruge de dyre Naade- Timer , at jeg altid med Frimodighed kan bie ester min Fraskillelses Stund og forvente i Haabet Guds-Borns herllge Frihed . Ak , Herre Iefu , kom , kom naadig , kom Ven god Stund ! I din Haand befaler jeg baade Legem og <b>Aand</b> , du forloste mig , Herre , du trofaste Gud . Amen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011061720001'target=_>Cyprianus, Thascius Caecilius, 1880, Om den almindelige Kirkes Enhed og om Herrens Bøn</a><h4>2126</h4>nied hint ? Dog , naar stal vi da besoge vore hsierestaaende Wner , hvis Gunst vi tiltrcenger ? Naar stal vi forberede os paa det , Disciplene betaler vs for ? Naar stal vi hvile v nd og styrke vor <b>Aand</b> efter de slavpende Bekymringer ? Mk med alt ! Lad os jage disse Forfcengeligheder og Wrskaber paa Dor , lad os vie os alene til Udforflning as 3 < mdheden ! Livet er elendigt ; Dsden er überegnelig . Om W pludselig kom over os , hvorledes vilde vi da gaa herfrå ? Wr skulde vi lcere , hvad vi her har forsomt ? Gllet macktte « i ikke meget mere bode haardt for denne Forssmmelighed ? tzwd , , om Dsden borttog og gjorde Ende paa alle Sorger mtidig med , at den bersver os Bevidstheden ? Det er ogsaa rt Spsrgsmaal . Men det er dog langt fra , at det er saa . Wer ikke for intet , at den kristelige Tro ndbreder sig med m saa overord entlig stor Anseelse over den ganske Jord , Aldrig bilde noget saa stort og herligt vcere blevet fuldbragt for os fil ! Guds Side , hvis ved Legemets Dsd ogsaa Sjcelens Liv gik tilgrunde . Hvorfor nsler vi da med at opgive vort Haab lil Verden og hengive os helt og holdent til at ssge Gud og et saligt Liv ? Dog , vent lidt ! Ogsaa de jordiske Ting er i ° g fornsielige ; de har en ikke ringe Ssdme , Man stal ikke l " telig afstjcrre sin Tilbsielighed for dem , efterdi det er en at komme tilbage til dem igjen . Se engang , hvor ei det ikke , at man opnaar Mre i en eller anden Hengde? Og hvor meget andet kunde der ikke vcere at snfle ? har en Mcengde fornemme Venner , Driver jeg kun ivrig " ° det ene og intet andet , kan jo endog et Statholderembede l > » e mig overdraget . Jeg kan faa mig en <b>Hustru</b> med mange saaat hun ikke forsger mine Udgifter ; og dermed vil ' " Begjcerligheds <b>Maal</b> blive fuldt . Mange store og fslgetende

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008091803008'target=_>Trollope, Frances, 1847, Constance Ridley</a><h4>616</h4>" O , det har ikke noget at betyde , min kjlrrc <b>Pige</b> , thi kan ikke det samme siges om de fleste af os ? Denne uanselige Form synes imidlertid at våreden jordiske Volig for to fuldkommeitt forskjellige Aandrr: Contrarius er t ^ iis og ss , obel ! g , crdelmodig og smaalig , Hui <b>Dannelse</b> og fiin Gratie synes at udgjsre hans <b>Sjcls</b> sande Kjerne — hans Phantasies Skabninger ere fulde af indtagrnde Skisnhedcr, hans Ltraben er reen , og dog vil han stundom nedladc sig til en ussel Hadefuldhed . og , l ' lot for at sige et spidst Ord ellrr en na ' rgaaende Lkjemt , gjsre sig merr berygtct for sin onde Tunge , end paassjonnet for sine gode Gjerninger . At hans Rltk er ffarpere end de fleste Mcends , vil ikke vare Dem paafaldende , og til Trods for hans Stemmes indtagende Blsdhed , kan man dog forlade sig paa , at denne om Voltaire mindende L ^ be er et paalidcligt Vidne om den Kja ' rlighed til spottende Ironie , der boer indcnfor . Men hvor vel tyder han os ikke , mcdens han sidder saaledes , som vi nu sce ham for os , lyttende til de phantaslilc Indfald , derudmales

<h4>2002</h4> Azura ! " svarcde den saaledes tiltalte <b>Dame</b> , i hiin sidonisse Tone , der fortener hver Stavelse et helt Vinds Indhold — . . Azura ! var det ikke fordi enhver Sammenligning er saa bcdragenfk , som den glattende Vind . der spotter Blomsten , som den leger med , kunde jeg fristes til at hentyde til Wdigheden afDercs egne silkeblsde , hsrgule Lotter og Frisshcden af Deres Hudfarve , der kappes med Hybenroscns Skjcrr , sammenlignet med hiin uslebne Nondepigcs , der netop passerede os forbi ; men lader os opgive en saa unyttig Undersogelse , og lad mig heller husvale mit altfor saaredc Hjerte ved at minde 3 em om , at den Kjcrrlighed , jeg ( i alt for fuldt <b>Maal</b> , vil 3 c maassee tcrnke ) har lagt for Dagcn ^ aldrig har vcrret af den Art . at en aandrig <b>Dame</b> ikt > ssulde kunne vedkjende sig den . Vi literaire Folk , min bedste Azura , maae ikke , kunne ikke og blive i Virkeliahedcn heller ikke bedomte efter de samme Love , som gjelde for alle andre Klassers selssavrlige Samliv . Nilar en <b>Dame</b> er begavet med en lys <b>Aand</b> og et modtageligt Gemyt . der hos Andre tilbeder det Talent , hun selv i saa

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008090303003'target=_>Møller, Poul, 1843, Efterladte Skrifter</a><h4>108</h4>Saaledes var han traadt ud i Verden paa en hccdcrlig Maade . — Det er en lykkelig Tid , disse Dage , da den Unge fsrste Gang tager Friheden i Besiddelse ; men i sit Hjerte bar han en dybcre Folclse , der ofte drog hans Langsler hen til den By , hvis Skolebccnkc han nylig havde forladt . Det var den friste , hellige Kjarlighed lil en <b>Pige</b> , hvis Billede gjorde hans Tanker til sin rcncste Glorie og fyldte dem mcd Jubel baade tidlig og silde . Han var endnu Skolediscipcl, hun nccstcn Barn , — da havde han scct hende , talt med hende , ofte , stundom daglig , og optaget hendes Vasen i sin inderste Tilucrrelse ; men fsrst et Par Aar efter fik han af hendes Mund Forvisning og Tilsagn om hvad han lange havde bcsiddct i Haabet . Dette var det <b>Formaal</b> , som bestandig kaldte ham tilbage til Vramcnsflio , nåar saa mange Ting lokkede hans livlige <b>Aand</b> fra Collegieheftcrne , saa at han uagtct mangen Adspredelse , dog fuldendte sine Cursus ved Universitetet til den sccdvanlige Tid . At kunne fsrc sin Elskede hjem som Brud , var altid Endepunktet for hans Onstcr og Forhaabninger . I Aaret 1813 underkastede han sig den anden Eramen , ved hvilken han blev udmcrrkct eller , som det hedder , indkaldt , og lagde sig derpaa efter Theologien , stjsndt han allerede dengang havde meest Tilbsiclighcd til at blive ved dc gamle Sprog og gaae den philologiste Vei .

<h4>662</h4>Til disse gunstige Tegn for Fremtiden kom ogsaa en gledelig Forandring i Poul Mollers huuslige Stilling , da han i Julen 1836 eglede Frsken v. Vulov , dcr havde veret hans Born en trofast <b>Moder</b> og var blcven ham selv dyrebar . Men det horer til det Dybe , Uforklarlige i Skjebnens Beslutninger , som ofte tager sig ud for vort Blik som en sonderknuscnde Ironie , at Naturen , samtidig med Saamegct , hvortil et frcidigt Livshaab kunde knytte sig , havde forberedct en Vending , der snart skulde fylde os alle med den bittrestc Smerte . Poul Mollcr havde allerede i nogen Tid jevnlig lidt af Stakaandethcd , og det var et foruroligende Varsel for hans Venner , nåar den kraftige Mand maatte bede dem gaae langsommere paa en Spadscrctour , eller nåar han maatte holde sig ved et Tre , medens han hentede Vciret . Men i 1837 kom disse asthmatiste Tilfalde hyppigere , og omsider maatte Legen kaldcs til . Det var nu ikke lenger af et frit Valg , nåar Poul Mollcr leste og arbeidede mindre ; han kjempede lenge mod Sygdommen og vilde ikke give tabt ; men endelig maatte han trekke sig tilbage fra alle Forretninger , og Ovcrhoringcn ved den philosophistc Eramcn i Eftcraarct 1837 , som han kun med Moie udforte , var hans sidste academiste Function I saadanne Tider , hvor Legemets Svaghcd afbrydcr Livets sedvanlige Gang , kan Aandcn endnu have Krefter tilbage , og Musen har Intet imot » at afleggc Sygcbesog . Saaledes kom hun endnu engang til Poul Moller — det var i samme Efteraar — da han forstemt og mismodig bctragtede den politiske Gjering i hans Fedrcncland . Men seer ikke sjelden Mennesker mcd hoi <b>aandclig</b> <b>Uddannelse</b> blive til Aristokrater , idet de hylde Tankens Suprematie igjennem enkelte utmerkede Individer og bctragte dcn ovrige Masse kun som Nul vcd Nul . De ville , at de Viseste stulle regjere . Dcsudcn havde Poul Moller fra Barn af veret vant til stor Nespcct mod dcn hoicre staaendc Personlighet » ; overfor cn Fader , som hans , maatte saadanne Folelser blive til Natur , og han fornegtede

<h4>1742</h4>Landets fjerne Klipper frembed . Lysgrsnne og merkegronne Stove bedcettede Fjeldene , saavidt vort Vie kunde rakke . Til Afverling laae hist og her sorte Klipper i Havet , og over dem gik en hoi Vranding . En af de foran Landet liggende Klipper havde ved et underligt Naturspil faaet Skabning af en stor Triumphbue . Ovenpaa den groede Tråer , og igjennem den saae man det stovbevorede Helland i Baggrunden . Et andet tratbedattet Vlalid var ogsaa af Naturen gjennemboret , men havde et langere Gjennemsnit , saa at det snarere var at ligne ved en stor Buegang af Steen , der stod hvalvet over Havet . Dersom disse hcengcnde Stove havre varet dannede ved Kunst , da vilde Semiramis ' s hcengende Haver i Sammenligning med dem have tabt deres Rang blcmdt den gamle Verdens Undervcerter . Men det vatter dog meer Forundring nu , da man veed , at de skylde deres <b>Dannelse</b> til dell vilde Natur . ( Athos-Bjerget , udhugget ved Alerander , kunde maasiee her bruges som Indledning ) * ) . Fra Landet tom vegetabilske Dunster os imsde , der vare desto fristere , da det Dagen tilforn havde regnet stcerkt . Den aromatiske Duft virkede paa vore Nerver , som om vi vandrede igjennem Haver af lutter lasminer . Jeg horte ogsaa fortalte , at en <b>vis</b> Plante , der i Lugt har Lighed med lasminen , stal vore meget hyppig paa Java . Batavias <b>Damer</b> bruge den til at pynte deres Hovedhaar . Da det lakkede mod Aften , blev Luften stedse meer og meer krydret . Men Landet forsvandt for vore Vine . Luften og Havet var atter rundt om os , og Merkets Dedsstille blev tun afbrudt ved den hule Dundren af Brandingen , der bred sig mod den fjerne Landbred .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009090403028'target=_>Wang, J. Storm, 1860, Skjebnen og mennesket</a><h4>850</h4>Vi gjensamles saalldes her med Madam Host , efter at flere Aar ere henrundne , dog var hun - ndnu uagtet sin ssrgelige Skjebne og de Aar , der ligge imellem , en smuk henved midaldrende <b>Kone</b> , drr udmcrrkVde sig ved sin fine <b>Dannelse</b> og liltrak sig iscrr Menncstekjenderens Hjerte ved dm hoitidclige , stille Kummer , der altid var udbrrdt over hende . Hoormeget eller hvorlidet hun pas « sede for de vindmgsse Lirkler , kom ikke til Spsrgsmaal om , da hun aldrig fsgtt Nogen udenfor det streiste Hus , og selv her var hun aldrig at overtale til at <b>vise</b> sig , naar der paa visse aarlige Festdage eller ved scrregne Anledninger gjordes Honnsrs , blot i det stille , daglige Liv , naar hun saa sig nsdsaget til for sine huslige Indretnlngers Skyld at trcrnge til den altid med hende velvillige Skrei , saacs hun simpel og net nnd sin lille , nydelige Linda ved Ha ^ ndrn at begive sig derhen , og som et velvilligt Offer for de Opofrelser og dlt Vcnstab , s > m den ans ^ te Kjsbmand viste hmde , lod hun sig da overtale til at forblive Aftenen dlandt hans Familie , til hvis enkelte Medlemmer hun virkelig maatte l < egge det største Baand paa sig selv , forat nedstemme sig , vaade for selv at forstaaes, og forat kunn ? fatte dcns Samtale , der i AlmindcUghed kun dreiede sig om Stedets Nyheder , der for hende vare saa aldeles intelsigende , at de aldrig fcrstede sig i hcndcs Erindring . Oftere derimod til visse beleilige Tider af Dagen gjorde Kjobmand Skrei hende selv sin Opvartnins , og i den senere Tid syntes han endog

<h4>853</h4>Madam Host havde viet sig til ene og alene at vcrre <b>Moder</b> , og dette Hverv opfyldte hun i strengeste Betydning, thi aldrig kunde noget Barn blive omhyggeligere opdraget og dannet end hendes lille Linda . Under den ommeste Moderkjcrrlighcd og blot som til Leg og Tidsfordriv gjmneingik hun Barnets fsrste og for det Meste stsrste Vanstelighed hcndes Abc , og siden uddanncdes Barnets <b>Sjel</b> i forskjellige Retninger . Moderen besnd selv <b>Kundskaber</b> i ethvert Fag , der egner sig for et Fruentimmcrs <b>Dannelse</b> , og aldrig kunde det falde hende ind at åndetro den kjerre Lille til nogen Skole eller Institut der paa Stedet , uagcet hendes Velynder Skrei i dette ene Punkt aldeles var uenig med hende og ssgte ved alle mulige Overtalclsesgrunde at bevcrge hende til at lade den Lille nyde Frugterne af de vindingste Opdragelsesanstalter, som han stildrede med de ssjonneste og livligste Farver, ja endog velvillige » selv tilbod sig at betale enhver Udgift , som dermed maatte vcere forbunden . At imidlertid Barnet ved dette Intet tabte , lunde Enhver blive overbevist om , der onskede at gjore sig bekjendt med bendes Kundstaber i forskjellige Fag , thi da Linda var sine ti til tolv Aar , lcrste og talede hun fiydende flere levende Sprog , foruden at bun i Musik og Tegnekunsten virkeligcn havde bragt det saa vidt , at Enhver , der horte hende og saa hendes Arbeider , maatte forbauses ; dog , det var ikke for Fremmede , Barnets Talenter bleve satte tilstue , det var alene det omme Moderhjerte , der i den stille og enlige Vraa frydede sig over dem . Man maa ikke tro , at Barnets Tid udelukkende offredes til den hoiere <b>Dannelse</b> , tverlimod med ligesaa stor Noiagtighed

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012041108157'target=_>Overskou, Th., 1860, Den danske Skueplads, i dens Historie, fra de første Spor af danske Skuespil indtil vor Tid</a><h4>5810</h4>Ven til ai bestemme sig for Mad . Nosmg , som , ester deres Mening , vilde give Nollen ined storrc as Charakteerskildringen . Da ogsaa Nosing onstede , at hans <b>Kone</b> her maattc sinde Leilighed til i en ny og jaa detvdelig Nolle at optrede i et Fag , hvorsor han vidste at l ' i , i ; bavde stor kunstnerisk <b>Begavelse</b> , gav Tamsoe ester . Slv ! ! , skulde imidiertid . som bestemt til Kongens Fodselofest , nodvendigviis gaae paa en fastiat Aslen , og Mad . forestaaende Nedtomst gjorde det meget tvivlsomt o , u bun paa den vilde kunne spille i ifolge Samsoes Onske dlev del dersor as Directionen befalet Mad . Preisler at indstudere Nollen som Double . Delte var rvgtedes i Publikum o . ) vakle stort Mionoie alleredc inden Opfovelsen . De fleste ventede sig mere as Viad . Preioler . som havde ei as Naturen majestetisk Udvortes , en suldtonende Talestenune og el mcget udtrykssuldt Blik , eud as Nosing , der vco <b>Aandsdannelse</b> maatte erstatte Viangelen as dioie store plni ' iskc Evner , Tilligc vidste man. at i de sidste Aar , da begge Kunstnerinder havde begvndl al gaae over i eldre Anstandsroller . var Mad . Nosing ofle bleven soretrukket Mad . Preioler , og den Uret , man sandt at der va ; steel denne med Hcnsyn til Eigbrit , blev sremdevet ioin saamegel bittrere , da bnn var ansat til at doublere et Parne , der oprindeligt havde veret bestemt for bende , og som det var kundbart , al Mad . Nostng i alle Fald vildc for en lang Tid kun blive istand til at spille el Par Gange . ~ Hvad er det andet " , skrev Tode til ~ eud al man ret grangiveligt vil <b>vise</b> , at en Mad . Preisler nu ikkc meie skal boldes sor god til andet , end at bere Sledel ' ov Mad . Rostng ? Maae de dog sinde dem iat Grevinde Orstnas n ' , csterligc Udforerinde kommer ti ! ai sville Tigdrn

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009071401049'target=_>Jensen, P.A., 1855, Læsebog til Brug for vore Skolers nederste og mellemste Classer ved Undervisningen i Modersmaalet</a><h4>5166</h4>har laant mangen Gjenstand en Glands , som ikke tilhorte den , men kun udgik fra hans egen <b>Sjel</b> . Men jeg kunde minde dig om saa mange Toner , som du aldrig vil forglemme , som ofte ville vere paa dine Lebcr i din Glede og i dit Vemod ; jeg kunde fore dig hen til saa Meget , som hans usortrsdne Flid har samlet , og som sildige Slegter endnu ville benytte . Dog minder jeg nu hellere om det Hele og Stor < som Krsniken stal bevare , som ingen Misgunst stal fordunkle . Derfor var dmne Mands Liv saa merkverdigt , saa velgjorende , fordi det tidligen blev ham betroet at pleie <b>Dannelsens</b> edle Spire i vort Land , fordi Gud havde aabnet hans Leber , at han kunde lere de Ingre det Sprog , hvori de kunde tolke , hvad der blev dem givet at udsige , og det for saa stor en Deel var hans Toner , der vcdbleve at lyde i de Danskes Tale , selv efterat Alder og Mismod havde forstemt hans egen Harpe . Derfor var denne Mands Liv saa merkverdigt , saa velgjorende , fordi han tidligen havde lert at gjmnemtrenge sin Tale med sit Hjertes Flammer , fordi han megtigen kunde tende i de Unges Bryst en Lue , som overstraaler Egennyttens og Lidmstabernes skumle Glsd , som neres af Alt , hvad dcr er edelt og helligt , som brender for Gud og Pligt , for Konge og Fedreland , for <b>Hustru</b> og Born og det fredelige Hjem og det hellige Venskab ; fordi han forstod at vette i dem det frie Sind , som hader enhver Treldom , hvilket Navn den endog berer , det Sind , i hvilket alene Trostab og Mod og edel Anstrengelse kan boe . Hvad han lette os ? — dct var ilke enkelte Lensetninger , han pregede i Sindet ; men han bidrog megtigen til at indgyde os den Folelse , uden hvilken Lere og Bedrift , og Lyst og Smerte og Liv og Dod kun ere et tomt , forfengeligt Spil .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006082900070'target=_>Brun, J.N., 1877, Gammelt nyt om &amp; af biskop Johan Nordahl Brun</a><h4>502</h4>Komedier " * ) . Tet siger sig selv , at saadanue Slutninger alene kan have sin Rod i Ensidighet ) , forenet med Übckjendtskab til Bruns Liv og Virksomhcd forovrigt . Med storre Ret tuudc man maaske med andre i Hensynet til de Tidsforhold og den By , hvori Brun virkede , finde en <b>vis</b> Berettigelse for hans Forhold i sidst omtalte Henseende .  Tet gjaldt vist dengang endnu mere end nu , at Bergen var en Stad , hvor alles Interesser dreicde sig om Handel forst og Handel sidst . For en aandig Strceben var der dcngang saarc lidcn Plads , og en saadan blev kun tarvelig fremhjulpcn ved de tvende Skoler , hvori Byens Ungdom nod sin Undervisning . Ncrften 20 Aar as dette Aarhundredc hcngik , inden Bergen havde noget ordentligt Pigcinstitut , og den Tanke var den herskende , at den unge <b>Kvinde</b> havde <b>Loerdom</b> nok , naar hun var indviet i Kjskkencts og Kjcldcrens Mysterier . At en Mand som Nordal Brun snart har faaet Viet op for den herskende Materialisme , er klart , og at han ogsaa i de Estetiske Interesser har scet et Middel til at hawc Befolkningen , er rimcligt . Hvorvidt han virkelig har havt dette <b>Maal</b> og naaet det , faar staa derhen , men sikkert er det vel , at han maa betragtcs som Boerer for al a åndelig Strceben i den By , hvor han virkede som den store Auktoritet i alle aandeligc Sager . " ( Thrap ) .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006082800062'target=_>Aubert, E., 1849, Om mundtlig Rettergang og Edsvorne</a><h4>2489</h4>fandt Hyrden . Man troe ilke , at der fremkom specielle Beviser til hans Forsvar ; det eneste Vidne i Sagen var Notaren , og Frifindclstn paafulgte blot paa Grund af Hyrdens Personlighet » . Uden ligefrcm at vare enfoldig , var han dog fuldkommen verfaren i alle det borgerlige Livs Forretninger . Han havde aldrig vceret udcnfor sin Landsby , var meget fattig , idet han om Sommeren vogtede Kvceg og om Vinteren fortjente sig Noget ved Daglon . Han havde aldrig besiddet fast Eiendom og aldrig overdraget en saadan , havde alrrig nogensinde vcerct hos en Notar , og vidste Intet om , hvorledes det gaaer til ved saadanne Forretninger . Bondekonen var den rigcste i Landsbyen , allsaa for ham at anste som en fornem Person , der paa Grund af hendcs Formue havdc meget at raade over . Han var vant til hendes Befalinger ; thi han havde ofte arbeidet i hendcs Huus og derved lagtMcerke til , at hun fsrte Kommandoen i Huset , og at alle Anordninger i Huusholdningen , saavelsom alle Dispositioner over Formuen udgik fra hende . DeEcdsvorne antoge derfor , at enMand , der var 65 Aar gammel og i al sin Tid havde tjent med Troskab og arbeidet med Nedclighed , paa sit Livs Aften ikke vilde besvcrre sin Samvittighet » mcd en stor Forbrydelse , og for en übetydelig Belonning ikke havre villet ovcrtage en Criminalstraf , hvis Ende han efter alle menneskelige Beregninger ilke vilde oplcve . De sluttede hcraf , at Manden blot havdc ladet sig bruge som Voerktoi , og havde handlet uden <b>Bevidsthed</b> om at have bcgaaet nogen Uret ( det er uden bedragerisk Hensigt) idet han havde anseet sin Deltagelse i det falske Document for en Tjeneste , som han var den <b>Kone</b> skyldig , der saa ofte havde givet ham Arbeide og Fortjeneste , og som enhver anden Daglonner ligesaagodt havde kunnet ydc mod Betaling . Jeg tenker , at denne <b>vise</b> Dom vil tilfredsstille endog de strengeste og rorc mange menneskelig siudede Criminalister . Men hvorfra skulde de kongelige Dommere kunne have en saa noie <b>Kundskab</b> til de laveste Stcenders Tcenke- og Handlemaade , Dannelsestrin og hele Vcrsen , hvoraf denne Dom alene har sit Udspring ? Ganske vist vilde enhver Dommer , hvor mundtlige Forhandlinger ere indforte , ligesaavel have frifundet den Anklagede , naar alle de Forudscetningcr , som den Eedsvorne allerede kjender fra det borgerlige kiv , udtrykkelig i Sagen vare blevne

<h4>2505</h4>senere hen har indgaaet i et alvorligt meent , og derefter er bleven sat under Tiltale for Bigami efter Art . 340 af Ca6 < » penal , der herfor foreskriver en Straf af tidsbestemt Tvangsarbeide ( fra 5 til 20 Aar ) . Her seer man af dette Erempel , hvilken uhyre Strid der kan vcere imellem den positive Lov og Folkets Toenkemaade og Religion. Kirkens Bud og Geistlighedens Loerdomme ere for Katholikerne af den laveste Folkeklasse den eneste Regel for deres Handlemaade, og naar han har handlet og levet overeensstemmende med denne sin Tro , er han fuldkommen <b>vis</b> paa den evige Salighed . Hans Tillid til Geistligheden er uden Grcrndser og den yngste Capellan har mere Magt over ham end den hsieste verdslige Autoritet*). Den katholske Kirke kjender intet borgerlig ! ZEgteskab , og foragter det som en Opsindelse , der kan blive farlig for Kirkens Indfiydelse og selv forfore til Frafald fra dens Bud . Naar den geistlige Samvittigheds-Raad i et borgerligt Wgteskab ingen Grund seer til gjensidige Pligter , saa seer Skriftebarnct det ligesaa lidet , thi dette veed og troer ikke mere end hiin siger . Jeg er overbeviist om , at der endnu iblandt <b>Fruentimmer</b> af den laveste Stand ikke gives en eneste , hvem det er bekjendt eller hvem det blot synes tcenkeligt , at en Handling , som for hende selv og hendes Skriftesader Intet er uden en blot og bar Formalitet , lovmcessigen er Mgteskabet selv , og at Gjentagelsen af en saadan Handling danner en stor Forbrydelse . ... I saadanne Tilfcelde have de Eedsvorne altid frifundet de Anklagede formedelst disses reliqiose Overbeviisning og formedelst deres Uvidenhed og Vildfarelse i Henseende til den positive Lov — og sikkert med fuldkommen Ret . Thi de vide vel , at ethvert Menneske er fri for moralsk Vrsde , naar han har handlet oprigtigt og efter sin Ovcrbeviisning og ikke med Vidende og Forsoet har feilet ; de lade sig ikke indproeke , at en i de foreskrevne Former bekjendtgjort Lov derfor ogsaa er bleven Enhver bekjendt , og deres Retsfolelse er endnu ikke ved <b>Loerdom</b> bleven saameget fordcervct , at de skulde kunne bringe moralsk Skyldfrihed og lovmcessig Straf i Forening med hinanden. Hvad har <b>Konen</b> derimod at vente af kongelige Dommere ?

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010020412003'target=_>Sundt, Eilert, 1850, Beretning om Fante- eller Landstrygerfolket i Norge</a><h4>45</h4>Som denne <b>Kvinde</b> gaar der en hel Slcegt i vort Land . Det er jo et gammelt bibelsk Villede at betegne et Folk som en <b>Kvinde</b> , og denne sirobclige , udlevede <b>Kvinde</b> er det rette Villede paa denne Slcegt . Som hun har den ogsaa engang scet noget bedre Dage , hvori den vist var lystig og glad og med let Sind betraadte Livets vanskelige Vei ; men nu er Slcegten meget gammel , og der har lagt sig Slovhed over dens Erindring , saa den kan ikke give nogen sammenhengende Forklaring om , hvorfra den stammer , og hvad dens <b>Maal</b> og Med har vcerct i Livet . Synd og Vrode bedcekker dens Aasyn og har der udstettet de Trcek vel ikke af Inde , men dog af Kvikhed og livligt Mod , som engang monne have vceret at skue . Den har aabcnbar udlcvet for lcenge siden , og det ser ud , som om ' Herren just derfor havde opholdt dens hjemlsse og hjcelpelose Liv indtil nu , fordi han vilde lade det sees , om der er villig Deltagelse i Landet for den dybestc Elendighed , hvori Mennesker kunne synke , om Kjcerlighcden eier <b>Visdom</b> til at berede Slcegten et Vederkvcegclsens Vad og lade et nyt Livs Trost skinne ned i dens stove og formsrkcde <b>Aand</b> .

<h4>502</h4>i Christendom . Det heddcr vel i Lovbestcmmclserne , at denne pderste Tvang fsrst bsr anvendes , naar vedkommende Prcest forgjceves har anvendt sine Formaninger ; men denne nu henved halvhundredaarige Mand havde nok aldrig om saadanne Ting vceret tiltalt af nogen Prcest eller anden Christcn ; han befandtes endog at vcere udobt . I Tugthuscts Prcest fandt hm : en saare venlig og omhyggelig Sjelesorger ; men hvor Tugthuslivet blev ham pinagtigt alligevel ! Han maatte begynde med ABC . At lcere at lcest er en vanskelig Kunst allerede for Smaaborn , der dog saa vidunderlig let lcere at tale det kunstige Sprog ; men trefold vansteligt maa det vcere for en Mand , som fra sin Ungdom af kun har ovet sig i Omstreiferlivets Nstadigheder og dertil er kommen ind i den Alder , hvor <b>Sjelen</b> allerede er mere uskikket til at lcere noget Nyt . Og dertil kom , at Mandcn havde en betydelig udviklet Stcer paa begge Vinene ! Der kom En ind i Skolestuen og hsrte ham nctop stave : U-b üb , u-f uf ! og saalcdcs gik det med Suk og Uf den hele Tid . Han bcgyndte endelig med Kathechismen ; men de underlige Vogstaver lagde sig som et Ncet for hans omtaagede Vlik og hindrcdc ham i at se og fatte den hoie Lcere , som laa bag Vogstaverne ; saa tcenkte han paa de mange Aar , den vanskelige <b>Lcerdom</b> vel vilde tåge fra hans Frihcd , og han stirrede paany med anstrcmgt Sands i Bogen ; men Dinene lob i Vand , og han kastede Bogen i Gulvet ; hans Kammerater trostede , og opmuntrede ham , og han forsogte atter ; men atter lob det i Et for ham , og han trampcde i Gulvet og skreg hsit af Fortvivlelse . Derhos maatte han tcenke paa , at hans  Kvindfolk " ( det konventionelle Navn for en <b>Hustru</b> i naturligt Mgtestab ) nu gik ude paa de vide Veie , maaste i fremmede Fanters Sclskab , hun , som dog kanffe i mange Aar havde bevaret ham sin Tro . Omtrent 5 Fjerdingaar vårede dette ; saa kunde han Katechismens 5 Parter tarveligt udenad , og dermed dobtes han . Han kncelede for Alteret og aftagde Loftet ; han boiede sit Hoved over Fonten , og den hellige^Handling fuldbyrdedes ; men ligesaalidt nu som for sporedes der i hans Mine og Holdning noget Tegn til levende Erkjendclse

<h4>1000</h4>thi han hav ikke ovct sig saaledes i at arbeide for et godt <b>Formaal</b> og gloede sig ved Nydelscn i Haabet ; Haabets Evne selv , den hoiere Forestillings Magt , er saa overmaade svag hos ham ; han tramger til mere umiddelbar Nydelse og Adspredelse under Ensformigheten ; hans stove Sind soger hyppigere og bestandig nye Pirrcmidler , og dem sinder han lettest i en uafladclig Stedsforandring med dens Vcrel af altid nye Forholde . For at undgaa et let Arbeides Ensformighet » og Kjedsomhcd vcelger han heller lange og tunge Dagmarschers Meie . Tiden falder ham saa lang , naar han er i Nolighed ; han trykkes og ovcrvceldes af Tidens Magt , og villie- og tankclost glider han ud i Rummet . Ved hver Times Arbeide vinder den Flittige et Tillceg til sin Lykke ; hans Hjertefred skyder en ny Rod , hans Haab udfolder et nyt Blad ; men den stove Omstreifers Hjerte har aldrig smagt Fred , og hans <b>Aand</b> kjendcr ikke Haabcts Opmuntring , og for hver Mil , han paa ilende Fod tilbagelcegger , soger han , ledet af et ulykkeligt Instinkt, at flygte for de nu og da opvaagnende alvorlige Tanker og end yderligcre at  udstnkke Aanden . " Derfor ser man ham gjerne ile mere rastlos , jo celdre han bliver , og jo tungere vel de morke Tanker ruge over ham . Etsteds i Nordlandene havde en Fattigkommission forbarmet sig over en gammel og skrobelig Omstrciferkvinde og skaffet hende et Lcegdslcms Ro , og i Nordlandene ved man at det er besvcerligt at foerdes enten til Lands eller til Vands ; men <b>Kvinden</b> var alligevel glad , da hun efter nogen Tids Forlob fandt Leilighed til i Folge med et Fantcselstab , som just kom forbi , i al Hemmelighet » at stryge afsted og begynde det besvcerlige Reiseliv igjen . Folk i Bygden forundrede sig storlig over Mennesket ; men det , de forundrede sig over , var her vist meget naturligt .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014061108124'target=_>Petersen, N.M., 1871, Bidrag til den danske literaturs historie</a><h4>3360</h4>indtil de kunde finde Noglerne til det ; saa at en Del saa for lidet , andre for meget deri . Fabelen var efter nogles Mening smukt og sindrig opfunden , men dog altfor usandsynlig . Dem svarer kan , at de kavde da ikke lagt Mærke til , kvorledes det just er Fabelens <b>Formaal</b> at gjore sig lystig over visse Forfattere , som i deres Beskrivelser over fjærne Lande kave bundet os mauge Æventyr paa Ærmet . Naar nogle tilstaa , at der var en viss Opfindsomked og grundige Lærdomme deri , men ogsaa adskilligt Fjas , der ikke somme de sig for en Olding og Filosof , saa minder han dem om , at de lærdeste , retsindigste og ærværdigste Mænd ofte have skrevet sligt , og Cicero siger , at seiv i Skjemt skinner et ædelt Gemyt igjennem . Skriftets Sigte i Almindelighed er at <b>vise</b> , hvorledes de fleste Mennesker gribe Skyggen for Legemet , og at hvad der kaldes sommeligt og usommeligt grunder sig mere paa Skikke og Sædvaner , end paa Tingenes egen Natur . Det ses fornemmelig af Klims Rejse om Planeten Nazar , som er Kjærnen af Skriftet . Hovedlærdommen sigter til et bestride den Vildfarelse , som gemenlig begaas i de Domme , der fældes over visse Personers Kapacitet og Dygtigbed til Embeder; man sætter mere Pris paa hastigt Begreb og Flygtighed , end paa langsom Skjonsomked . For en stor Del har han ogsaa skrevet denne underjordiske Rejse , for at oplyse , at nåar vi betragte vore egne Skikke og <b>Moder</b> , og Ugne dem med deres , som vi kalde barbariske , have vi kun liden Ære at tale med. Det er ikke , som nogle have bebrejdet ham , af Knarvurrenhed han kar belet de fleste europæiske Skikke og Vedtægter . Sigtet er herved heller at forsvare og undskylde andre Nationers , som vi seiv bele og foragte , alene efterdi de ere ikke overensstemmende med vore egne ; de saakaldte barbariske Nationer kan finde ligesaa frugtbar Materie til Latter endog i vore mest polerede Sæder . Skriftet sigter i saa Maade keiler til at undskylde end til at vrage Sædvaner , og til at erindre dem , som bele fremmede Vedtægter , at undersøge med Upartiskhed og efter den sunde Fornuft deres egne . Da ville de finde , at de <b>Moder</b> , som de have forelsket sig i , og som de holde for de de mest rimelige og sirlige , grunde sig alene derpaa , at de ere offenlig komne i Brug og almindelig antagne . Naar vi komme i fremmede Lande , gaar det de fleste af os ligesom Niels Klim , , hvilken paa sine Rejser holdt alt hvad han saa for vauskabt . Ved denne Lejligked erindrer han , hvorledes han en Gang rejste med nogle franske <b>Fruentimmer</b> fra Paris til Holland ; i det første Herberg de kom i Braband , brast de i Latter , fordi Pigen strøede Sand paa Gulvet , hvilket er noget ganske almindeligt i de nordiske Lande . Ligeledes , da en norsk Bonde saa en Gang Rigens Amiral Ove Gjedde drikke Tke og en anden Gang æde

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014111408013'target=_>Wexels, W.A., 1846, Udsigt over Jesu Christi Liv paa Jorden</a><h4>93</h4>vor Gjerning har Fremgang , eller naar vi glemme os selv i Sysler og Nydelser og Adspredelser , det er der dog og bryder frem mangen Stund , ofte netop i den , vi kalde vor lykkeligste ; ja , jo dybere , jo inderligere og sandere det Smiil er , hvormed vi tolke Glæden i denne eller hiin Gave , der blev os til Deel , des klarere bliver Taaren os , som hcenger ved Smilet , des klarere den hemmelige Smerte , som er i Glccden , desmere faacr denne Vemodens Boesen ; og , mene vi Sukket qvalt ved Selvforglemmelse , ved at overgive Sjelcn og Hjertet enten til Det , der stiller vor S andseligheds Krav , til „ Viets Lyst og Kjodets Lyst og et hoffærdigt Levnet , " eller til Det , som lover Ncering sor vor Tanke , til Grublen , til verdslig Viisdom , til Opdagelse af og Syslen med , hvad der kan tjene til Gavn ellerForstjonnelse for det jordiske Liv i det borgerlige eller huuslige Samfund , — , da feile vi , og Sukket vil vaagne , naar vi mindst vente det , og Smerten vil gane gjennem vor <b>Sjel</b> , naar vi troe den . lcengst borte . Men netop fordi Menneskene saa gjerne ville dolge hiint Suk for sig selv , og dove det , saa godt de kunne , netop derfor soge de ikke Forklaring deraf , Forklaring af , hvad der dog itte lader sig adskille fra deres Vcesen , og hvad de dog i enkelte Timer maae hore uden- og indenfra . Selv naar de nodes til at blive sig Sukket klart bevidste , til at sige sig selv : „ Jeg loengcs übeskrivelig efter Noget , jeg savner Noget ; der cr Noget , jeg maa sinde , hvis jeg sial blive glad , hvis min <b>Sjel</b> stal faac Fred , " selv da lade de det ofte beroe ved dette übestemte „ Noget , " sporge ikke : „ hvad favner jeg ? hvad loengcs jeg efter ? hvad cr det , jeg maa finde , for at min Uro kan stilles , min Smerte ophore ? " Og dog ligger Svaret saa ncer : det Snk , der gjennem Verden gaaer , er et Suk efter Forlosning . og Lcenker og sukker efter Forlosning ; Mennesket er i Vaand og Loenker og sukker efter Forlosning ; efter denne sukker Legemet , efter denne sukker Aanden . Hvad cr det , som har bragt Mennesket selv og den Natur , i hvilken det har sit Liv , ja hvis Hjerte og Middelpunkt det er , i Vaand og Lcenker ? Det er Synden , det er Faldet fra Gud , Faldet / som stedte i Menneskeslægtens Fader og <b>Moder</b> , og trcengte fra dem gjennem den hele Slocgt . Kun i Samfund meo Gud er der for Skabningen Liv og Frihed , salig , uHemmet Udvikling af alle dens Kroefter , salig , uhindret Tilfredsstillelse af hver dens Trang og Attråa , salig , fuldendt Opnaaelse af dens <b>Maal</b> , dens Bestemmelse . Det faldne Menneske kom under onde acmdclige Magters

<h4>146</h4>Da Tidens Fylde kom , ' da den hellige Time var oprunden , som Gud fra Evighed havde forud bestemt , da Alt var forberedt , da Verdens Nod og Jammer var steget til det Hsieste og allevegne streg om Hjcelp , om Hjcelp fra Himmelen , da Tomheden af alle menneskelige Hjcelpe- og Troste-Kilder blev mere og mere aabenbar , da tusinde og atter tusinde Hjerter futtede og sionnede under det uhyre <b>Maal</b> af Fordærvelse , da Lcengselen efter en Frelser , da Anelsen , da Forventningen om den Begæredes Komme var vakt trindt omkring , da alle Tegn og Varsler forkyndte , at noget Forunderligt , noget Stort skulde siee , da „ sendte Gud sin Son fsdt af en <b>Qvinde</b> , " da „ blev Ordet , Faderens Ord , hans udtrykte Billede og hans Vcesens rette Afglands , Kjsd , og boede iblandt os , fuld af Naade og Sandhed, og « åbenbarede sin Herlighed , en Herlighed , som den Eenbaarncs af Faderen . " At Frelsen skulde udgaae af Abrahams Afkom , af Israels det udvalgte Folk , af luda Stamme , af Davids Slcegt , det var klarligen forkyndt , og i Bethlehem , Davids Stad , saae omtrent Aar efter Menneskets Skabelse han Lyset , hvis Navn blev kaldet Jesus , som det var kaldet af Engelen for han blev undfangen i Moders Liv , og denne Jesus , hvis Navn er bleven stort paa Jorderige, som Ingens , og hviler med Lovsang i Umyndiges Mund og paa Oldingens Lceber scia vidt omkring som Solen vandrer med sit Lys , denne Jesus , der i sit tredivte Aar fremstod som Prophet og gik om blandt sit Folk og lcerte og bekræftede sin Lcrre som guddommelig ved Tegn og Undergerninger , idet han gjorde vel og helbrededc Syge og opvakte Dode og uddrev onde Aander og prcedikede Evangelium for de Fattige , Lcegedom for de Sonderknuste , Losladelse for de Fangne , Syn for de Blinde , Frihed for de Plagede , Herrens velbehagelige Aar for hans Folk , denne Jesus , som , stjsndt Ingen kunde paavise nogen Synd hos ham , og stjsndt hans Vei var i Kjcerlighed og Ydmyghed og Sandhed , dog blev hadet af Folkets Vverstcr , af dem domt til Doden som en Gndsbespotter , overgivet til den romerske Landshovding, Pontius Pilatus , og ester hans Befaling naglet som en Forbryder til Korset , paa hvilket han udgav sin <b>Aand</b> , under sine Fienders

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015030308039'target=_>Wergeland, Henrik, 1857, Henrik Wergelands Samlede Skrifter</a><h4>2476</h4>Forlællingen om Morianerne blev afhrudt ved at Budejen , det første hun kom ind med Melken , raabte ; „ men Jøssu Namn , <b>Mor</b> , Grauten er sviji.lngen af dem , som havde hørt paa Skolemesteren , angrede paa at Groden var sveden . <b>Mor</b> gav et Stykke stegt Flesk tilbedste , som Skolemesteren fik for sin egen Mund , da han begyndte Maaltidet med den Bemærkning , at Trynetyrken plejede at anrette Kristenkjødet , som altid maatte være fedt , aldeles som Svinesteg . Man kunde kjende Landet alt langtfra paa Lugten o . s , v. Den første Hentydning var nok til at belage de andre Appetiten ; men Skolemesteren var baade en slærk <b>Aand</b> og en stærk Mave . laften fik han sidde i Højsædet , og befæstede derfrå endnu mere sin Anseelse ved nogle forfærdelige Bedrifter af „ Puntekorven “ , som fra en liden Dreng af blev til Konge og slog Bonaparten . Disse havde han fra en liden i Kristiania udkommen Bog , men sin anden <b>Lærdom</b> havde han fra Studeringer i sine yngre Dage , om hvis Vidtløftighed kun Den kunde gjøre sig et Begreb , som havde seet Bøger , der vare større end Bibelen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008092303003'target=_>Knigge, Adolph, 1857, Om Omgang med Mennesker</a><h4>1419</h4>for Alvor beherskede af den slettcrc <b>Aand</b> . Gjerrige , havesygc, dovne og krybendc , smigrende de Fornemme og Rige . stolte og overmodige mod de Ringere , hadcfulde og misundeligc mod deres Ligemcend , ere de for stsrstedelen Skyld i , om Ningeagt for den hellige Religion og dens Tjenere blwer saa almindelig udbredt . De be « tragte denne Religion som en tsr Videnskab , og deres Embede som en indbringendc Forretning . Paa Landet blive de bondcagtige , hcngivc sig til Lediggang og Magelighcd, og klage over , at de have et uhyre Arbejde , naar de eengang om Ugen bestige Proedikcstolen og dysse Tilhsrcrnc i Ssvn med deres dogmatiske , usle Spidsfindigheder. De angle ester Forcrringer , Arveparter og Legater , som Djcevlcn ester deres Sjcel . Deres Wrgjerrighed er umaadelig , deres gejstlige Stolthet » , De » ! potisme og kirkelige Herstesygc kjender ingen Grcendser . De bruge Religionsiver til Skjermbraldt for deres Lidcnskaber. Orthodoxi er deres Losen , blind Tro og Guds Wre er deres Feltraab . naar de ville forfolge den rolige, uskyldige Borger , som gjor Forstjel paa Religion og Theologi samt undlader at yde Smiger og Offer til dem . De ncere et uforsonligt Fjendstab — jeg taler af Erfaring — mod dem , der ikke ville underkaste sig deres lernscepter eller tie til deres Samvittighedslsshed . De ere forfoengeligcrc end en <b>Kvinde</b> . Bevaeget af Paatrcrngenhed og barnagtig Nysgjerrighcd snige de sig ind i Huse og Familier , for at blande sig i Ting . som ikke angaa dem . for at opspinde Rcenker , stifte Uenighet » og siste i rort Vande . Ingen forstaaer bedre end de den Kunst , paa en listig Maade at scette en Sag igjennem , idet de overvinde alle Vanskeligheter , uden at have Udsecnde as at have en Haand med i Spillet . Gaacr det ikke paa den ene Maade , begynde de paa den ganske modsatte , drejc . vende , besmykke , forrykke Synspunktet og hvile ikke . for de have naaet det <b>Maal</b> , de have sat sig , hvad enten det bestaaer i at tilfredsstille deres Herskesyge . Hevngjerrighed eller Havcsyge .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014072808037'target=_>Kurtz, Joh. Heinr., 1866, hellige historie</a><h4>970</h4>Dannelse . Saadanne Gnomedigtere findes ogsaa hos andre Folk , men den væsentlige Forstjel er denne , at de hebraiske <b>Vise</b> gaae ud fra den objective guddommelige Aabenbaring selv , og , idet de anvende sammes Sandheder paa det almindelige Livs Relationer og Forhold , ved dybsindig Overveielse og ' besjælede af den samme <b>Aand</b> , af hvilken Aabenbaringen felv er ndgaaet , opstille en i Sandhed hellig Livsviisdom. Ligesom David viser sig som den egentlige Ophavsmand til , og den meest fuldendte Mester i Psalmedigtmngen , saaledes er Salomon den egentlige Fader til og Mester : Ordsprogdigtningen , og hvad der er os opbevaret af den Art stammer uden Tvivl for stsrste Delen fra ham . An merk . Vor Ordsprogsamling bestaaer , som Overskrifterne <b>vise</b> , af flere Samlinger . De 9 fsrste Capitler udgjsre et Heelt , et Slags almindelig Deel , hvis Hovedindhold er Lovpriisning af Vnsdommen . Med det 10. Cap . begynder med Overskriften : „ Salomons Ordsprog " den egentlige Samling . En ny Samling indeholder Cav . 25 — 29 , som indledes ved de Ord : „ Ogsaa disse ere Salomons Ordsprog , hvilke Ezechias , luda Konges , Mcend have samlet " . Indtil her have vt altsaa kun Ordsprog af Salomon , der til forskjellige Tider ere blevne samlede . Cup . 30 indeholder Ordsprog af en anden Viis , ved Navn A g ur , og Cap . 31 begynder saaledes : „ Lemuels , Kongens Ord , ved hvilke hans <b>Moder</b> optugtede ham " , og flutter med alphabettfi ordnede Ordsprog til Lovpriisning af den vygtige Huustro . ( Lemuel er et fingeret Navn Gud hengiven . - Maafiee er Agur ogsaa Forfatter til denne Deel . )

<h4>1348</h4>1 ) Maria , den ocdle Jomfru af Davids kongelige Sla ' gt , fattig og ringe i Verdens Qine , men dyrebar og udvalgt for Herren , rig paa barnlig Idmyghed , sm Inderlighed og troende Hengivenhed , var den <b>Qvinde</b> , der var udkaaren til at vcere Frelserens <b>Moder</b> ( sml . § 129 , Anm . ) . I hende udfoldede sig Qvindelighedens fineste og cedleste Blomst , i hende blev Qvindens hsieste Bestemmelse til Virkelighed , derfor er ogsaa det hele Qvindekjon velsignet i hende . Hun er <b>Moder</b> til den anden Adam , med hvem den nye Udvikling af Menneskeslægten begyndte , Ojenbilledet af den forste <b>Qvinde</b> , en „ Livets <b>Moder</b> " i hoiere Forstand . Allerede i den lauge Rcekke af hendes Forfoedre havde Herren udvalgt hende ; hende gjaldt paa en <b>vis</b> Maade Abrahams Kaldelse , Davids Salvelse osv. ; thi den <b>Qvinde</b> , i hvis Moderstjsd Gud selv kunde antage meunestelig Natur og Skikkelse , var <b>Maalet</b> for hiin lange Rletke af Fodseler ( § 14 , Anm . ) , som , omendstjondt den var inddragen med i den menneskelige Syndigheds almindelige Forbandelse ( Ps . 5 ! , 7 ) , desuagtet , baaren af den guddommelige Raadslutnings Velsignelseslraft , ganer ligesom en gylden Kjcrde fra det ene Led til det andet gjennem den hele forchristelige Udvikling . Maria var det sidste Led i denne Kjcedc , i hvilket den blot menneskelige Fsdsel ovhorte og gjorde Plads for den umiddelbart guddommelige , den hele gammeltestamentlige Udvikling afsluttende Fsdsel .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008090503002'target=_>Becker, Karl Friedrich, 1841, Friedrich Becker's Verdenshistorie</a><h4>353</h4>De greske Helerer ere meget omtalte . Det blodagtige Lille-Asien , i Serdeleshed Milet , var disse Personers Fedrenelcmd og Planteskole ; de fleste og rigeste fandtes i Korinth , hvor de endog node et Slags Udmerkelse; de beromteste i Athenen . Allerede Solon , der ved strenge Straffelove imod Wgtesiabsbmd og Forforelse havde sogt at sikkre Borgerindernes Seder , taalte Hetereme og tilstod dem Beskyttelse , og , hvad et Fragment af en komisk Digter bevidner , i samme Hensigt , thi i Folge antike Begreber kom fremfor Alt Borgeme og deres Familier i Betragtning , men Hetereme vare Fremmede eller Ufrie , hvis moralske Tilverelse under alle Omstendigheter kunde opoffres til Bedste for hine . De sogte , i Serdeleshed i Athenen , ikke blot at glimre ved Skionhed , men ogsaa ved <b>Aand</b> og Charakteer . Det gielder iser om Asp asia , den beromteste af alle greske Helerer , hvem Perikles , efterat hun lenge havde veret hans Veninde , tilsidst endog egtede , og som havde en stor Indflydelse paa ham , af hvilken Gmnd begge derfor ikke heller undgik Komedieskrivernes bittre Spot . Hvor udmerket hendes <b>Aand</b> og hendes <b>Dannelse</b> maa have veret , seer man af Plato , der i Menerenus lader Sokrates sige , at hun har veret hans Lererinde / i Veltalenhet » , hun , som havde dannet mange andre store Talere , og ogsaa den storste af dem Alle , Perikles . Men Aspasia gjorde ogsaa en Undtagelse fra de ovnge <b>Fruentimmer</b> , til hvis Klasse hun blev regnet , en Undtagelse , som

<h4>2342</h4>Alligevel lod en ved <b>Aand</b> , Veltalenhet » og <b>Dannelse</b> ndmcerket Mand sig ikke afstrcrtte fra at betrcrde denne Vei . Det var Tib . Sempronius Grncchns , som vi allende have lårt at tiende fra ben numcmtinste Krig , eit Son af ben som Censor omtalte Gracchns , og en Dcttterson nf den aldre Scipio Africcmus . Haus <b>Moder</b> Co rn eli a , en <b>Qvinde</b> med ell hoi <b>Aand</b> , der var dannet ved den graste Litteratur og Philosophie , havde afslaaet en cegypist Konges Haand , for at knnne lede sine tvende Sonners Ovdragclse .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2007072603004'target=_>Blom, Gustav Peter, 1849, Fordum og Nu</a><h4>663</h4>Da Vogt havde endt sin Fortolling , sagde Berg : Ester de Forhold , hvori Anna befinder fig , og ester de cedle Forscetter , De har fattet med Hensyn til hende , sinder jeg ingen Vetcenkeligheder i , at De lader hende nyde fcelles Underviisning med Deres Datter , og det vil vcere mig en Glcrde , ved at bibringe hende de <b>Kundskaber</b> jeg formaaer , at bidrage min Skjcerv til den livlige <b>Piges</b> » Vel og Uafhcengighrd . — Jeg vil saaledes begynde den Underviisning , De har betroet mig , og forbeholder mig kun Deres faderlige Naad og Veiledning , naar jeg ncermere lcerer at kjende de kjerre <b>Pigers</b> <b>Aandsevner</b> og Charakteer .

<h4>943</h4>svarende Opdragelse . Jeg studerede ved Universitetet i Salamanca, og da jeg af Naturen var begavet med et godt Hoved , gjardc jeg god Fremgang . Allerede i mit 16 de Aar havde jeg erhvervrt mig de <b>Kundskaber</b> , der udfordres til at trcede ind i et geistligt Gmbede , men min levende <b>Aand</b> stmbte efter mere , og jeg formaaede min Fader til at tillade mig at fortsatte mine Studcringer , dem jeg udstrakte ikke blot til Theologi og de dsde Sprog , men ogsaa til Historie , Naturvidenskaber og Philosophi . Jeg tilegnede mig fiere fremmede Sprog og gjorde mig bekjendt med fremmede Landes Litteratur . Jo mere min <b>Kundskab</b> udvidede sig , desto mere befoestedes jeg i den Overbevisning , at den geistlige Stand ikke stemmede med min Tilbsielighrd og mine Anskuelser af Livet og Menneskets Bestemmelse. Jeg vovede imidlertid ikke at udtale mig aabent derom til min Fader , der var en stramg Mand , og vilde ansee dette som en Fornærmelse mod den faderlige Myndigycd , der i de spanske adelige Familier er stor . Jeg har imidlertid Grund til at troe , at han anede min Tilboielighed og ikke aldeles misbilligede den , thi han lod mig uforstyrret vedblive mine Studeringer , og l od flere Leiligheder til at anbringe mig i et geistligt Gmbede gaae übenyttede hen . Da jeg var 22 Aar gammel , dsde min Fader af et Slagstod , og da min <b>Moder</b> allrrede for var dsd , stod jeg alene i Verden, uden andon Stette end den , mine <b>Kundskaber</b> og en hsist übetydelig Formue kunde give mig . Jeg maatte nu bestemme mig for en Stilling i Livet . Min Formue var ikke tilstrekkelig til

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008110603024'target=_>Vig, Ole, 1859, Enkelte af Ole Vigs Arbeider til Folkelæsning</a><h4>92</h4>Lærerstanden maatte hceves til et hojere Standpunkt baade i <b>aandelig</b> og materiel Henseende , saa vidste han doq meget vel , at ligesaalidt Oprsr som unyttig Klynk var Vejen hvorpaa dette <b>Maal</b> skulde naaes . Standen maatte hceve sig selv , om den skulde blive agtet as Andre ; og dertil var Dygtighed i dens Gjerning , Eendrcegtighed og Broderaand de fortrinligste Midler . Men for at opnaa dette var det hojst nodvendigt , hvad der lcenge var blevm overset , at den fik et aabent Dje sor , at dens Virksomhed ikke danner en afsiuttet Kreds for sig selv , men staar og maa staa i levende Sammenhang med det hele Folk og med Alt , hvad der heder Skoler og Skolevcrsen i Landet . Derfor var det en af Vigs Hovedbcskjceftigelser i Kristiansund udenfor hans egen Skolevirksomhed og ovrige boglige Sysler at bringe Skolelcercrforeninger tillive rundt om i Landet, hvor Folk af alle Steender og Lcerere fra allestags Skoler kunde lomme sammen og samtale om denne fcelles for det hele Folks aandelige Udvikling saa - vigtige Gjerning. Han strev i den Anledning en Mcengde Breve til alle sine Bekjendte og Venner rundt om , iscer i det Norden og Vcstenfjeldste , hvor han havde sine steste Venstabsforbindelser, og opmuntrede dem til at lcegge alvorlig Haand paa Vcerket . " , , Maatte dog snart , " siger han ,  de forstjellige Klasser og Stcender , og saaledes ogsaa Almustolclcererne, i Norge lcere at indse , at til at virke noget rigtigt Stort og Godt til Fcrdrenelandets Held , er det nodvendigt at slutte sig sammen og ved <b>Moder</b> og Sammenkomster at lede de enkelte Krcefter til eet <b>Maal</b> , styrke Samfunds- , Enigheds- og Broderbaandct og igjennem det mundtlige Ord lade Tanker og Folelser utveksles ; . det vilde idetmindste blive glade og for hele Livet frugtbringende og ovlivende Timer . "

<h4>123</h4>Oplysuing endnu ikke har faat rigtig Ma.gt med den , dette Almusprog fandt endelig en ligesaa nidkjcer som dygtia Samler og ' Granster . Det var Folkeaandens Krcefter til at gjenfode sig selv , der saaledes i Slutningen af det forlobne halve Aarhundrede havde begyndt for Alvor at rore sig , og hvor saadanne Magter er ude , der behover de blot en Herold for at <b>vise</b> , hvortil de er bestemte , og hvad de mcegter at virke . Og det var vistnok ogsaa paa hoj Tid , der stede saadanne alvorlige Greb paa de gamle Strenge , hvis de ikke aldeles skulde tysne hen ; thi saavidt var det kommen med vort eget Modersmaal , at de Lcerde ikke engang bekymrede sig om at se endmindre tcenke derpaa , de saakaldte,,Dannede" skycde det som usle Pjalter , der vel kunde vcere gode nok at hange paa det stakkels Norges fattige Legeme , indtil vor gamle <b>Moder</b> engang maaske kunde blive ria , nok til at koste sig nye Helligdagsklceder efter fineste Parijermode med Snorliv og stive Fiskcbensfljorter . Synet for Folket som et a and el igt Helt med fcelles Fortid , Nutid og Fremtid , med fcelles Betingelser for en sund og kraftig Udvikling , var dels slovet , dels tabt , og det netop hos deM , af hvem der medrette skulde kunne forores Hjcelp i Nodens Tid . Om det end ikke lcenger var saa ilde som i gamle Holbergs Dage , at  Franst med sin Frue og Tyst med sin Hund og Dansk med sin Tjener man talte , " saa talte man dog ikke Norsk med nogen af Delene , og vilde vel i mange Henseender endnu givet den gode Holberg Anledning til en hjertestyrkende Latter . Det gjaldt snart sagt indenen <b>vis</b> Del af Folket at kunne spille Don Ranudo paa Verdensstuepladsen, med broget Flojel foran , Nogenhed bag og Blcekklatter , hvor Hunerne var forstore . Og det var ikke godt dette , thi hvor Dannelsen eller den saakaldte Intelligents," kun er en forloren Blomst sammensyd af laante Lapper , og ikke paa naturlig Maade udsprnngen af Nasjonens eget aandelige Livstrce , der bliver man kun til Latter for Andre og til lidcn Nytte for sig selv . Naar nu hertil kommer , at selv Almuen , der dog endnu for en stor Del ejte og ejer sit Modersmaal friskt og kraftigt og sin egen Tankegang ufordervet , ogsaa havde lcert saameget af Tidens Tone , at den ikke alene havde begyndt at foragte sit eget <b>Maal</b> , men endogsaa i hojeste Maade at forarges over den , der vilde tale eller skrive det ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013010208011'target=_>Kieler, Laura, 1869, Brands Døtre</a><h4>60</h4>SM , hvilket det desvccrre ikke er , som det burde vcere , mine <b>Damer</b> ; men en Profet bliver sjelden agtet i sit eget Fædreneland , en meget sørgelig Kjendsgjerning ! " og den talende , der selv var et Stykke af en Poet , sukkede dybt . „ Der er en Fylde af Tanker , en imponerende Kraft og Djcervhed i Udtrykkene , en Rigdom af Villeder , et Hav af Poesi , et Sprog , der er beundringsværdigt , og som alt man henrive enhver , der selv besidder en Smule poetisk <b>Dannelse</b> " . Hr. Kloumann havde her talt sig ganske varm . Fru Hall , den <b>Dame</b> der fMst havde taget Ordet , nikkede fmilende . „ Det var Sproget , Tankernes Form , den ydre Pragt og Poesi , men Tankerne selv , Hr. Kloumann , Indholdet ? " — „ Indholdet ? " den unge Herre lagde sine Arme overkors med en forbause : Mine . „ Indholdet ? " og han rynkede Panden med et knusende Alvor , som om han var den personificerede Kritik selv , „ Indholdet , Frue ! det er rcntud sagt noget Gallimathias " . — En foruroligende Vevcegelse hertes i Vccrelset . „ Hvad siger De ? Hr. Klonmann ! Gallimathias ! " raabte samtlige unge <b>Damer</b> indignerede , idet de stimlede sammen omkring ham . „ Vel , mine <b>Damer</b> , Gallimathias eller Galmandsideer " , sagde den tiltalte noget bestyrtset over dette uventede Angreb , „ men dette forringer van ingen Mande min Beundring sor Vcerkets digteriste SWnhed , ja , jeg « ilde endogsaa beundre det mindre , dersom der var mere sund Fornuft og ikke et faadant , jeg kan gjerne sige , opløftende Vanvid , der hcever Fantasien " „ En Digter har altsaa Privilegium paa at vcere gal , Hr. Kloumann ? " faldt en af de unge <b>Damer</b> ham leende i Talen . Hr. Kloumann bukkede . „ Ja . i en <b>vis</b> cesthetifl Forstand , Froken . Naar jeg lccser et Digt , Mster jeg ikke at finde belcerende Forelcesninger , men jeg Mster at fan min Indbildningskraft hcevet , min poetiske Sands tilfredsstillet, hvilket „ „ Brands " " Digters ildfulde <b>Aand</b> , overraskende , dristige Sprang fra den ene Tanke til den anden " „ Tor De virkelig foretage faadcmne himmelhpie , for Forstanden farlige Sprang fammen med en saadan Vanvittig , som „ „ Brands " "

<h4>1310</h4>der som et yndigt <b>Maal</b> for alles Blikke . I den sine Nodme og de klare Taarer , som perlede i de lange , Menhaar , leeste man hendes Bevægelse og hendes Lykke . Og der stod hun ved Siden af ham , den Mand , som mere end en misundte hende , som havde vceret Gjenstanden for manges Beundring , Drpmme og Forhaabninger . Med hpi Stemme aflagde han Liftet , der bandt ham for hele Livet til den unge , elskelige <b>Kvinde</b> ved hans Side , og da han rakte hende sin Haand , som siden mandigt og trofast , pint og sitert forte hende gjcnncm Glcrder og Sorger , gjennem Medgang og Modgang , og langt og taust fcestede sit Blit paa hende , da boiede Helga sit Hoved , og den ene Taare efter den anden faldt ned paa Kirkegulvet . I dette Blik saa hun en Borgen for Søsterens Fremtid , og med svulmende Hjerte takkede hun Gud . — Venligt indbydende og elegant serveret ventede Middagsbordet paa Nyb , o . Alle folte fig fornoiede og vel tilmode . Blid og opnnrrlsom havde Maria altid et indtagende Smil og et muntert Svar rede for enhver , men hendes Tankcr vare hos de to dyrebare , hvis Legemer hvilede derborte paa Kirlegaarden , men hvis frigjorte og salige vare i Guds Himmel . Over Doktorens Va ' sen og Tale var der ligeledes udbredt en <b>vis</b> Vemod , ogsaa han tamltc paa de bortgangne . Da anden Net kom frem , greb Fogden sit Glas og reiste sig op. Han var oiensynlig i en meget hoitidelig Slemning , og med bemrget Nost — forglemmende fin fcrdvanlige hoitravende Deklamation — teltede han i faa , men hjertelige Ord Menighedens Dufter for det unge Brudepar , bragte dem dens kærlige Hilsen og nedbad til Slutning Guds Velsignelse over Ording og hans <b>Hustru</b> . Hvad ham selv angik , skyldte han den unge Brudgom saa meget , at han ikke i en Staaltnle formanede at omtale det eller lade

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012101624012'target=_>Holberg, Ludvig, 1870, Holbergs Comedier</a><h4>4723</h4>Petronius sagte . De sige : Hvad har Saul at giore blant Propheter ? Jeg kunde jo svare med en <b>vis</b> <b>Dame</b> : Han ledte efter fin Herres Mer . Ha ha ha ! <b>Maalet</b> er ogfaa , gid jeg faae Skam , Pernilles . Ha ha ha !

<h4>4728</h4>19 - 20. jeg kunde jo svare med en <b>vis</b> <b>Dame</b> : Han ledte eter sin Herres 3 Esler ) Hvem denne <b>Dame</b> var , som gjorde en v : - ig Anvendelse af det ZErende , at opssge nogle Msler , i hvilket Sanl va draget ud , dengang han traf Samuel og kom ind i en Skare Propher ( 1 Sam . 9 - 10 ) , derom kan her Intet siges . — 20 - 21 , <b>Maalet</b> er ogv , . . Pernilles Malet , Stemmen .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016101248023'target=_>, 1852, Forordninger, aabne Breve, Placater m. m. for kongeriget Norge i Tidsrummet fra 1648-1813</a><h4>4730</h4>maa og fra l lan . 18 li af ingen Prcest foretage nogen Copulation eller Wgteviclse, med mindre de Eopulerrde , saavel Brudgom som Brud bcviisliggjore . at de cre vaccinerede , eller have haft de naturlige Kopper . 11. Naar de. naturlige eller Vsrnekopper udbryde nogetsteds paa bandet i de . ssgl . Niger , bor samtlige Beboere af den eller de Byer , hvor Smitten har viist sig . der ittc beviisligen have havt enten disse Kopper eller Kokopperne , uden Undtagelse strar underkastes Vaecinationcn . 12 For samtlige Kjobsteeder derimod in > strcrnlcs denne Befaling derhen , at Alle og Enhver i de Huse , hvor de naturlige Kopper yttrc sig , stal værc pligtige strar at lade sig vaccinere , med mindre de have havt de naturlige eller Kokopperne . 13. Laa stal ogsaa de Huse eller Gaarde i Kjobstcederne . i hvilkc Koppesygdommcn maatte <b>vise</b> sig . strar giores kjendelige derved , at en trykt eller skreven Seddel , som indeholder de Ord ' „ Her er Bornekoppcr " , paaklistrrs Gadcdorcn eller Porten , og lige Sedler paahcrftcs alle Indgangne til den families Bcerclscr , hos hvilken den Syge opholder sig ; og haver paa ethvert Sted nsie at overholde , at disse Sedler blive staaende urorte . indtil Districtslergcn . ogiKjobruhavnStadsphysicus. skriftlige « tillader deres Nedtagelse . Et saadant Huus , hvor Kopperne have yttret sig , bor iovrigt , uagtet den Syge endog bortfores fra samme , fremdeles ansecs som misccenkt for Koppesmitte . og Stedets Lcrge drage Omsorg for at de nodoendige Ncnsningsmidlcr , saasom mineralsse Nogelser o . s . v. der afbenyttes , samt jevnligcn undersvge . om Sygdommen der videre skulde udbrede sig . 14. Da de naturlige Kopper undertiden vanskeligen kan kjendes fra andre Hudsygdomme , saa bor ethvert Udslet , som bestaarr af smaa Blegne , og som viser sig strar efter en mere cllcr mindre kjendelig Sygdom , ansees som mistccnkeligt for at verre Kop « per , og bør dcsaarsag anmeldes paa den i W 15 og 16 befalede Maade . under de deri fastsatte Straffe i Overtra ' delses-Tilfcelde . naar Sygdommen siden befindes at være Kopper- 15. Enhver Huusfader eller <b>Huusmoder</b> skal , saasnart Tegn til Boruekoppcr <b>vise</b> sig i deres Huse , ufortovet gjore Anmeldelse derom , paa bandet for Slcdets Prccst eller Sognefogden , om han boer nccrmere ved Stedet , hvilkc igjcn skal anmelde det for Districtslcegen og Politiemesteren , som strar har at indberette det til Amtmanden ; i Kjobsta ' derne for Byfogden eller Stedets Leege . og i Kjøbenhavn for Politiemesteren eller Stadsphysuus . paa det at de fornodne Foranstaltninger mod Sygdommen strar kan vorde foiede . Mangel af en saadan Anmeldelse paadragrr den skyldige Huusfader eller <b>Huusmoder</b> Straf fra 8 Dages Fcengsel vaa Band og Brod indtil 3 Maancdcrs Arbcide i Forbedringshuset efter Forseelsens Beskaffenhed og Domstolenes Skjon . 16. ligeledes skal det og i Kjobsta ' derne værc Pligt for enhver Huusvert . eller den , som i hans Sted har Opsyn i Husel > noie at underrette sig om , og uopholdelige « for Districtslcegen at anmelde dennc Sygdoms Udbrud . Befindes han at have efterladt saadan Anmeldelse, uagtet Sygdommen har været ham bekjendt , straffes han med Mulct fra 10 Ndlr . til 100 Rdlr . cftcr hans Vilkaar og Forscclscns Grad . Som Følge hcraf bcrrttiges Huusva ' rterne til enhver Undersogelsr , hvorved de kan sccttes i Stand til at opfylde den dcm paaliggende Pligt . 17. Saasnart Lcrgerne , enten paa den i fornccvncc §§ befalede Maade , eller og ved egen Erfaring komme til <b>Kundskab</b> om Koppcsygdommes Udbrud , skal de. foruden at indberette det til Amtmanden , uopholdelige « give den vedkommende Stifts- eller Landphysicus saadant tilkjende ; ligesom de ogsaa i dette Tilfalde strar bor gjore Indberetning til Commissionen for Vaccinationen . 18. De Forsigtighedsregler , som ved Frr . 1 ? April 1782 for Danmark og 5 Srptbr . 1704 for Norge erc foreskrevne i Henseende til andre epidemiske Sygdomme , stal ogsaa værc udvidede til Koppesygdommen , førsaavidt ilke ved dennc Anordning anderledes er bestemt > ) . Disse Forsigtighedsregler ere : at

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111103044'target=_>Lamartine, Alphonse de, 1847, Girondisternes Historie</a><h4>354</h4>Kongen vendte tilbage til sit Verelse og lukkede sig inde med sin Skriftefader , Abbee Hodert , for at luttre fm <b>Sjel</b> og ostre sit Blod ; Dronningen og Prindsessen bleve ene med deres <b>Damer</b> . Da Prindsesse Elisabeth aftog sit Halstsrklede , inden hun lagde sig paa Sofaen , loste huu en Agrafe af Carneol , som hun havde i Brystet og hvori den fromme <b>Qvinde</b> havde ladet indgrave Ordene : „ Glemsel for Fornermelser , Tilgivelse for Krenkelser . " „ leg frygter , " sagde hun smilende , „ at denne <b>Lerdom</b> kun bliver en Sandhed for os . Men den er ikke destomindre en guddommelig Forskrift og bor derfor vere os ikke mindre hellig . " Dronningen lod de <b>Damer</b> , hun holdt meest af , sidde ved sine Fodder . De to Fyrstinder " kunde ikke sove . De talte hvidstende med hinanden om det Skrekkelige i deres

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011071312001'target=_>Samarow, Gregor, 1880, Kronprinsens Regiment</a><h4>96</h4>som han at drage store Acmder til sig , og med deres Hjcelp ogsaa at skaffe den gode Smag Indgang hos Publikum ? — Man taler jo saa meget om Kronprinsen af Preussen , hans fine <b>Dannelse</b> og hans rige <b>Aand</b> — ham er det maaste forbcholdt ikke blot at efterligne den franske Hofetikette , men ogsaa i Tyskland at skaffe Forstaaelfe for den franske Nations store <b>Aandsliv</b> " . Wngstelig saa von Bielfeld sig om . ~ Tal ikke saa om Kronprinsen , min kjcere Ven " , sagde han , ~ man kan aldrig vide , af hvem et i en faa tcet Trcengsel udtalt Ord kan opsnappes , og det kunde vcere farligt for den stakkars Prins , hvis et saadant til hans tilkommende Regjering sat Haab blev hort paa urette Sted " . Deres Samtale blev afbrudt af lydelige Stemmer i en heftig Ordvekfel , hvilke trcengte frem af en tcet Kreds , som havde dannet sig foran Boden med Lykkehjulet . Man hsrte en kvindekg Stemmes cengstelige Raab , indimellem lydelig Latter og igjen uvillig truende Mumlen . ~ Hsrer De " , raabte Chevalieren , ~ der drives der et lidet Mventyr , og det synes som om man trner en <b>Kvinde</b> , — Ridder- Pligten byder at gaa derhen og i Nsdsfald at yde de svage Bistand " . Hr. von Bielfeld syntes lidet fornsjet over denne Opbrusen af ridderlig Fslelse . Dog , Hr. Montbar

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009071303016'target=_>Bornemann, Frederik Christian, 1863, Samlede skrifter</a><h4>1635</h4>Vi have i det Forcgaaende feet , at Communismen ikke blot medfsrer Eiendommens men ogsaa Familiens Ophcevelse . Anstille vi nu paa dette Punct en Sammenligning mellem Socialismen og den rene Communisme , saa finde vi vel , at nogle Socialister bestemt angribe Wgtestabet , om de endog ville lade det bestaae af Navn ; men andre afvise disse Angreb , og ville ikke erkjende , at de folge af deres Principer . Man maa nu ogsaa tilstaae , at den dyrifke Promiscuitet , som ligefrem felger af den egentlige Communismes Forudscetninger , ingenlunde stemmer med Socialismens <b>Aand</b> . Ligesaalidet som Socialisten vil give Afkald paa det forfinede Liv , det forsgede materielle Velvcere , som kun det civiliserede Liv byder , ligesaalidet vil han lade sig nsie med den raae sandselige Nydelse i Kjonsomgjcengelsen ; han fordrer Kicerlighedsnydelse . Men efter den hele Opfattelse afMenneftets Natur og Bestemmelse , der er Socialismens Forudscrtning , dens Forncegtelse eller Ikke - Erkjendelse af den menneskelige Frihed , Miskjendclsen af Menneskets Bestemmelse til med Overvindelse af de naturlige Drivter , af den syndige Natur at arbeide for scedeliqe <b>Formaal</b> , i Stedet for hvilken Socialisterne scette Menneskets Bestemmelse i Nydelsen , i Tilfredsstillelsen af de naturlige Drivter , som uden Videre ere gode , — maa Socialismen i sit inderste Vcrsen vccre en vistnok mindre raa , men just derfor endnu farligere Fjende . En saa alvorlig Sag som en KMsforbindelse . i hvilken man under Sorg og Modgang , under alle Omskiftelser trofast stal holde ud , i hvilken det ikke er tilladt at sige : min , eller vor Tilboielighed har nu taget en anden Retning , nu har jeg fundet en anden Gjenstand , hvis Besiddelse vilde forskjsnne mit Liv , — en saadan Kjsnsforbindelse er ikke Socialismens Sag , den maa for den vcere uden al Berettigelse . Er Wgtestabet foreneligt med Socialismen , saa er det i al Fald ikke det livsvarige LEgteskab . Og fremdeles , naar een Mand eller Qvinde

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013102108116'target=_>Jersin, Jens Dinesen, 1878, Livets sande Vei og Troens Kamp og Seier</a><h4>59</h4>Af Erfaring har jeg ogsaa befundet , at det ikke er , som Somme mens , at denne <b>Lcerdom</b> er for dyb og mprk for de Enfoldige og derfor utjenlig for dem . Thi vel maa jeg erkjende , at der sindes Noget deriblandt , som gaar over de Enfoldiges Forstand og kommer dem underligt for , fom endnu kun ere vante til Melken ( Ebr . 5 , 13 > og ikke have forsogt den haarde Fode , de , gjennem hvis Hjerte det Svcerd endnu ikke har trcrngt , som den gode gamle Simeon spaar den fromme , glade Jomfru Maria , Jesu Christi <b>Moder</b> om ( Luc . 2 ) ( denne Spaadom gjelder ogsaa alle dem , der <b>aandelig</b> boere den Herre lesum i Hjerterne og derfor ere ham med saadan <b>moderlig</b> Kjcerlighed forbundne ) ; men jeg har dog ogfaa befundet , at de Enfoldige og de , som enfoldigst kjende Christum og ikke ved mere end sit Modersmaal , ofte have de dybeste Anfegtelser og derfor allermest trcrnge til denne Undervisning. Derfor har jeg ogsaa skrevet den vaa vort danske Sprog , stjont jeg med mindre Umage og storre Anseelse ( men Gud forbyde , at jeg skulde soge den ; det , jeg strider, stjcenkes mig dog af hans Naade alene ) kunde have

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100803012'target=_>, 1847, Rescripter, Resolutioner og Collegial-Breve for Kongeriget Norge i Tidsrummet fra 1660-1813</a><h4>776</h4>pillcticum , som i det Hele vidncr om <b>Kundskab</b> i Tingen ' ) ; li ) den anden Charactcer udfordrer en mere udstrakt Kundstab saavcl i alle Hoved- , som de fleste Forbercdclscs- og Hjalpc- Vidcnsiabcr , hvilke navngives i den nye Fundation Caft . 4 88 15 og 16 ; o ) den ferste Charactccr kan alene forhvcrvcs ved fortrinlig Kundstab , hvis ei i alle , saa dog i de fleste lil Lovtyndighcden henhørende Vidcnsiaber ; mangelagtig Kundstab i en eller anden Hjalpe-Vidcnstab fan i og for sig selv endnu ikke betage nogm denne Chavacteer , men aldeles rigtig og grundig <b>Kundskab</b> in nntnilv , ifentiuin ot pul ) ! icc ) umvei-snli , tilstralkclig Kundstab i > > iiiic civili linmlmn , udbrcdt theorctist og praclisk Indsigt in . juro clnnico uoi vo ^ icn ot > i > ' jvnt < is og neiagtig <b>Kundskab</b> om Fadrclandcts Statsforfatning ere forudcn et fortrinligt Bpeoimon pincticum til Charactcren absolut nrdvcndige . § 12. Den enhver Candidat ved Pluraliteten tilkjendtc Charactecr indfores i Distinctions-Protocollcn , og delte underskrives af de tilforordnede Assessorer saavelsom Examinatorcrne . Her staacr det og enhver af disse Sidste ftit for i alt Fald at optegne sin ViBB6nBli3 fra Pluraliteten , og Grundene derfor , hvilken s > iBseii3UB ligcledcs med alle deus Grunde neiagtigcn sial indforcs i den Beretning , som Facultctct har at indstndc til Patronen over Udfaldet af hver Examen . 8 13. Dersom nogen Candidat i en eller anden Deel af de examinerendcs Vidensiaber sardeles har udmarket sig ( hvorhen ogsaa horer , om Nogen har profitcrct pudlicum Kel-mlmieum ) , stal det i Ovcrcensstcmmclsc med Fundatsens Cap . 4 8 1 ? anmarkcs i Distinctions-Protocollcn, og indfores i den mcddclcndcs Attest , som undersirives af Lxl , mln » tc ) i ibuB , og parapheres af I ^ c > tl » i-io kncuitntiz . § 14. Endstjendt Universitetets Larcre , der ere Examiuatores ved den juridiske Examen , have Ret og Frihet » til ved alle Slags Forelasningcr og Dvelser , som Vxlnninlltm-ia , I ) iBput : > tui ' in , og desligc , at berede og danne de Studerende til at <b>vise</b> Kundstab og Fardighcd in Lxamiue er dog den egentlige saa kaldte Manuduction , langt mindre et forud anstille ! ' leulnmeii , dem ikke tilladt , men de have deri fremdeles , ligesom hidindtil , at holde sig fra Alt , hvad der kunde have Skin af nogen Partisthed . § 15. Efter forcstaacnde ndforligc Forskrift for den latinske juridiske Examen bchoves ingen sardeles for den Danske men alene en noicre Bestemmelse af nogle enkelte Poster . Hvad altsaa angaaer det opgivendes specimen practimim , Tilsynet med Udarbeidelscn , og Maaden paa hvilken Preven skal forctages , da maae alle de i ovenstaacnde 3 die til ste § forestrevne Regler her anvendes og iagttagcs . Ved den mundtlige Examen kan det her vare nok , naal 2 Examinatorer prove Candidaten , men alligevel bor , for Censurens Skyld tre vare derved narvarende , og tåge Dccl i Censuren . Til Prevelsc admittcrcs 4 Candidatcr tillige paa cen Dag , og hver Examinator , som efter sin Tour cxaminercr , tilstaacs dertil N Time ; men vil nogen af de ovrige Examinatorer udenfor sin Orden tåge Deel deri , maa han ei dertil bruge mere end til , ^ Time . I Henseende til Protocollen , og hvad deri sial indfrres gjalder her 8 10. Til Chcn-acteren b e q v e m udfordres Kundstab i Natur- og Folkc-Rettcn , saavidt den med Billighet » fordres af Ustuderede , saa god thcoretifk Indsigt i den Danske og Norske Lovkyndighed, som uden Hjcelpcvidenstaber kan forhverves , og et vel forfattet Bpecimon p > «cticum; til Characteren ei übeqvem , nogenledes Kundstab i Natur-Rctten , temmelig god theorctisk Kundsiab i den Danstc og Norske Ret , og et skikkelig forfattet B > i < ? cimen s > i-n < : ticum . § 16. Naar viBB < znBUB ved Censuren <b>moder</b> , forholdes ester 8 12. — I ovrigt forbliver det ved de Forskrifter , som allerede i sidste Anordning om den juridiske Examen ere fastsatte , og overladcs til Vedkommendes Indsigt og Samvittighet » , paa bcdste Maade at ovfylde denne Examens Hensigt lil sandt Gavn og Bcdste for Staten , Videnstaberne ' og Lovkvndighcden .

<h4>4282</h4>Veleb , og vedkommende Fattige gives da Anviisnings-Seblcr til be Steder , hvor , til-saadan Udlevcring , Anstalt er feiet , imod at disse Sedler siden derfra med rede Penge for Fattig- Vasencts Regning indloses , og imod at Fattig-Vascnct forvisser sig om , at det Anviste lommer den Fattige selv og ingen Anden tilgode . 9 ) Den Deel af Almissens Pengc-Bcleb , der tilstaacs som Huusleic , udbetalcs Verterne til visse aftaltc Tider fra Fattig Vascnet . < l > ) For saavidt fattige Born ikte ved egne Foraldre , Slagt eller Nenner lunne forsorgen , bcserger Fattig-Vascnct dem ved den Undcrstetning , som for ethvert sindcs nedvcndig , op < fostrede hos egne Foraldre , eller hos gode Pleieforeldre . § 38. Saa mange Spisekvarterer , som Directionen anseer fornodcnt , indrcttes ftaa de belciligstc Steder , hvorfra de Fattige , som enten ere saa uordentlige , at dem ei kan betrocs Penge , eller hvis huusligc Omstandigheder ere saalcdcs beskafnc , at de selv ei med tilbellig Occonomic kunne skaffe sig ligcsaa sund Fedc , som Fattig-Vcesenets Indretning frcmbydcr , for en <b>vis</b> Priis lunne forsvinn lil alle Aarets Tider med de mccst narcnde , sunde og billige Fodemidler . V . Gin Skoie-Vcrseuet stir fattige Sgrn . § 39. Alle Born i Staden , som ikke have Foraldre eller Formyndere , der kunne , eller Andre , der ville betale for deres Skolegang , bor gives fri Nnderviisning , og Fattig-Vasencts Direclion bor drage Omsorg for og medvirke til , at de nytte samme . 8 49. Saasnart Dircttioncn for Fattig-Vascnct er traadt sammen , scctter den sig i Correspondence med Dircctionen over det Kongsbergsie Skole- Fond ' ) , samt vedkommende Kgl . Departements , for at faae saa mange af de Skoler , Fondet hidtil har vedligeholdct , som dehovcs til de fattige Berns Undcrviisning , sig overdragnc tilligemed de fornodne Indtagter til at bestride Udgifterne ved Undervisningen , samt til at bevirke andre Forandringer ved Skolc-Vascnct , som maatte ansees nedvendigc eller gavnligc for Fattig-Vascnets Diemed . § - 41. Skolcgangens Dicmcd bor vare : at Vernene ved deres <b>Sjels</b> og Legems Krafters rigtige Ndvilling og Anvendelse kunne dannes til oplvstc , gode , arbeidsomme Mennesker og Borgere , at de til den Ende tidligcn hindres fra Lediggang 04 Laster , og vannes til Sadeligbed , Orden og Neenlighcd ; at de anferes til saadan Kundstab, som er nodvcndig og gavnlig for Mennesket og Borgeren i Almindelighed ; og at de undervises og oves i saadant Arbeide , hvorved de i Fremtiden , enten i Mangel af andet Narings-Arbeide kunde fortjene deres Underholdning , eller nyttigcn beskaftigc sig i ledige Timer . § 42. Undervisningen i Skolernc stal altsaa vare deelt imcllem Videnskab , Haand- Arbeide , og nyttige Legcms-Wvelser . Z 43. Ved Videnstab forstaacs ikke her det , som henherei til blot <b>Lardom</b> , eller blot speculativ <b>Kundskab</b> i egentlig Forstand , men det , som for den menige Nlmue er fornodcut , fatteligt og brugbart , saasom : Lasning , Skrivning , Regning, Religion og Moral , almindclig Kundftab om Naturen , dens Indretninger og Virkninger; om borgerlige Indretningcr og Pligter , om de vigtigste Sundheds-Negler , om Fadrclandets Forfatning og Beliggcnhcd og deslige , alt efter korte og simple som dertil kunne valgcs , og med Hensyn til den Forstjcl , som ber gjercs i Underviisning for Drcngc og <b>Piger</b> . Ogsaa sorgcs for , at Bornene anferes til rigtig og harmonisk Sang , isar Chora » ler og gode Folkcsange . § 44. Haand » Nrbcidet bor vare af det Slags , som i det daglige Liv lan for Almuens Folk vare almindeligt og gavnligt , og som for Bernene ester Forsijcllighcd af Kjen , Alder og Krafter findes passende , samt hvortil i Skolebygningane kan vare Leilighed og Rum , saasom : at pille Bomuld , at karde Uld og Bomuld . , at plukke og spindc Uld , Bomuld , Hor , Hamp og-Blaar , at tvindc , haspe , vinde , spole , at binde og strikke , at lniple , at lappe paa Klader , at stoppe Strempcr , at knytte Fiskegarn , at vave Bandlcr og uldnc Vaand , at gjere Hagtcr og Fiskekroge , flette Kurve og Hatte , at forfardige allchaande Legetei og andet Nurnbcrger-Kram , og andet mere ; og for Pigcrnc tilligc , at tildannc og sve Linned og Klader , m . v. Ved Arbeidets nfvexlendc Afdeling sees blandt andet ogsaa dcrpaa, at Legems ' Krafternc styrkes tilligcmcd oves . § 45. Til at give Undervisning i Haand-Arbcide ansattes ved hver af Fatlig-Vasenets Skoler en Larcmoder , som anvises den fornedne Plads i Skolebygningen ; og forsaavidt Laremoderen ei kunde gjere det , soger Directionen at forene sig med ecn eller flere Arbeidere , som maatte besidde de fornednc Fardighedcr, om visse Timer daglig enten hjemme hos sig selv eller i Skolebygningerne at give Undcrviisning i Haand-Arbcide . § 46. Legems-Dvelser bor vare af det Slags , som kunde bidrage til den Styrke , Smidighet » og Behandighed , som i den arbeidende Classc ere nedvendige og nyttige . Drengebornenc ber om Sommeren ofte fores ud at bade sig i Laugen , og . tilligc gives Underviisning iat svomme . For at dette med Sitkerhcd kan siee , traffes de fornrdne Foranstaltninger . Til at undervise i Lcgems-Dvclser antages helst en duelig Under-Officcer , som er oplart i Gymnastikcn , ba han tillige kunde eve de aldre Drengebern iat elercerc . § Ingen Understettclse fra Fattig-Vascnct givcs til Underholdning for fattige Vern , uden under det udtrytkclige Vilkaar , at Vernene skulle stadigcn sege den Un <

<h4>4646</h4>han sinder forneden ved nogen Fattigs Begravelse , om saadan Familie end Mc fer har nvdt offentlig Understettelse . 85. Fjerde Cl as se , eller Lagds-Lcmmcr . DaLagde-Indretningen') er , ved ethvert Fatligvasen paa Landet , saavel den rigeste og hcnsigtsmassigste , som for Bygden kostbareste Hjalpckilde , saa fortjener denne af saavel Commission som Inspecteurer bet noiagtigste Tilsyn . De Regler , hvorefter den bliver at bestyre , ere selgende : 1 ) Forsergclse ved Lagd in natura anvises ikke Andre , end Bern over 9 Aars Alder , ber nu kunne gjere Tjeneste som Gjeterdreng eller Barncpige , og Saadannc , der af Alderdom ikke ganffe kunne forsorgc sig selv , men dog endnu have Krafter tilovers til nogen Nytte i Livet . 2 ) Naar altsaa det forhen borttingede Barn har fyldt sit 9 de Aar , oversattes det i 4 de Classe eller blandt Lagdslemmerne . Dog kan og bor heri stcc Undtagclscr ester Forstjellighcdcn af Barnets saavel Helbred som Legemsstorrclse , saaledes at det baade tidligere kan udtages af Iste Classe , og vel ogsaa langere forblive der . Ligcledes tages det Pengebeleb i Betragtning , for hvilket Barnet er borttinget , saaledes , at naar der hos retsiafne Folk , der ligcsaa omhvggeligen , som dette vilde stec hos Lagdsbonden , serge saa vel for dets Legems Pleie som <b>Sjels</b> <b>Dannelse</b> , end at lade det nytte Bygdens almindelige Skoleundervisning , kan forblive til mindre Udgift for Cassen , end ved at anvise det Lagd , saa forbliver det saa lange i Iste Classe ; og ferger da Commissiomn i Forening med Distnctcts Inspccteur omhyggeligen for , at det i 10 , 11 Aar eller hoierc og lavere Alder , dog altid under 15 Aar , da det bor consirmcres og forserge sig selv , kan faae ' Tjenesteplads hos brave Folk , saa det kan übgaae af Casfen . 3 ) Naar derimod den aldre Fattige , der er anuiist Lagd in natura , ved tiltagende Alderdom og Skrebelighcd bliver saa ussel , at han slet ilke kan gjere Nytte hos Lagdsbonden , men tvertimod forvolder ham alt for stor Vyrde , hvilket Inspecteuren skjenner og foredragcr Commissionen , da t , ^ ges han ud af Lagdct og overflyttes i Iste Classe eller blandt de Borttingede . 4 ) Ved de Fattiges Nnviisning til Lagd in natura tages deres Alder og Helbred i Betragtning , saaledes at de ikke altid faae fuld Gaard , men overensstemmende med deres Krafter forlaggcs paa H eller Gaarde , og ved tiltagende Alder og Krafters Svatkclse omtaggcs siden paa fuld Gaard , indtil de gaae ganske ud af Lagds- Cassen og over ide Borttingedcs . 5 ) Da det er befundet , at adsiillige enkelte Gamle , Manb saavelsom <b>Qvinder</b> , hellere modtage et mindre Bidrag i Penge end den Foregclse , man kunde anvise dem ved Lagd in natura , saa kan vel Commissionen tilstaae dem saadant , men under strengeste Forftligtelse at ikke betle , og noiagtig at opgive for Districtets Inspccteur, hvor og hvorved de have deres fornedne Ophold . Naar de denmod ilke langere lunne lade sig neie med bemeldte mindre Bidrag af Cassen , men enste Lagd in natura , da melde de sig derom betimeligcn hos Inspecteurcn indcn Omlagnings-Commissionen , der inbtraffcr i Juni og Dcccmbcr Mcaneber , paa det denne , der bor have en Fortegnelse over alle til Districtet henherende Gaarde , med hosfeiet Oplysning , enten de ere belagte eller ledige , kan opgive for Commissionen den Gaard i Districtet , som han kjender passende til den Ansegendes Helbredsforfatning , eller , i Mangel af ledige Gaarde i eget Distnct , fordre en saadan i et af de andre . 6 ) Born forblive ikke langcre under offentlig Forsorgelsc , altsaa ilke heller som Lagd paa nogen Gaard , end indtil de ere consirmercde ; og da dette , efter bcherig Skolegang, kan og bor siee i den sidste Halvdecl af deres 15 de Aar , saa serger saavcl Lagbsbonden, som Inspecteuren , i Forening med Commissionen , i dette sidste Lagdeaar omhyggcligcn for , at fnae dem udstadede i Tjeneste strax cfler Consirmationen . Til Erstatning for de Consirmations- og andre Klader , Lagdsbonden har bekostet , og som ved Udtradclsen af Lagdel tilhore Barnet , bliver hans Gaard fri for Bidrag indtil nastc Commission ; hvorimod han dog ogsaa i samme Tid beholder Barnet , saafremt det ikke er lykkedes at skaffe det Tjeneste. 7 ) Naar nogen Fattig , og fornemmelige » Bern , bliver anviist Lagd paa flere Smaagaarde sammentagne , saaledes , at de , ester en <b>vis</b> Tids Ophold paa hver isar , stulle stytte fra den ene til den anden , da stmbcr Commissionen heller at faac dem forblivende paa den ene af saadanne Gaarde , imod at tilstaae denne de andres Contingent i Penge . Dette Bcleb udbetales da af Cassen , og de Gaarde , der skulde have havt Lagdslemmct til Forsorgclse in natur » , indbetale igjen til Casscn . 8 ) Naar den Fattige er anviist Lagd in natura , da er det ham aldeles forbudet , paa egen Haand at contrahere med Lagdsbondcn om at nyde Penge eller Levnetsmidler uden Huset imod at give Slip paa det tilstaaede Ophold paa Gaarden. Saadant maa alene jkce med Inspccteurens Vidende og Samtykke , der da stuller en ordentlig Contract imcllcm Lagdsbonden og Lagdslemmet , og paaseer , at denne nyder det Fornedne til Livets Ophold . Ned saadan Contract ber dog , saavidt muligt , sorges for , at den Fattige ilke kommer ud af Districtet , og aldeles ikke ud af Bygden , paa det Inspecteuren kan have ham under nodvendigt Tilsyn . 9 ) Til Forsergelse in natura herer : fornoden Underholdning, lige med Gaardens andre Tjenere , Klaber , forsaavidt den Fattige ikke dermed

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008090903001'target=_>Sundt, Eilert, 1852, Beretning om Fante- eller Landstrygerfolket i Norge</a><h4>45</h4>Som deune <b>Kvinde</b> gaar der en hel Slegt i vort Land . Det er jo et gammelt bibelsk Billede at betegne et Folk som en <b>Kvinde</b> . og hin sirobelige , udlevcde Kviudc er det rette . Billede paa denne Slegt . Som hun har den ogsaa engang seet noget bedre Dage , hvori den vist var lystig og glad og med let Sind betraadte Livets vanskelige Vei ; men nu er Slegien meget gammel , og der . har lagt sig Slovhed over dens Erindring , saa den kan ikke give nogen sammenhengende Forklaring om , hvorfra den stammer , og hvad dens <b>Maal</b> og Med har veret i Livet . Synd og Brode bedekker dens Aasyn og har der udslettet de Trek vel ikke af Dnde , men dog af Kvikhed og livligt Mod , som engang monne have veret at stue . Den har aabcnbar udlevet for lenge siden , og det seer ud , som om Herren just derfor havde opholdt dens hjemlose og hjelpelosc Liv indtil nu , fordi han vilde lade det sees , om der er villig Deltagelse i Landet for den dybeste Elendighed , hvori Mennesker kunne synke , om Kjerligheden eier <b>Visdom</b> til at berede Slcegten et Vederkvegelsens Bad og lade et nyt Livs Trost stinne ned i dens stove og formerkede <b>Aand</b> .

<h4>497</h4>i Christendom . Det hedder vel i Lovbesiemmclserne , at denne yderste Tvang forst bor anvendes , nåar vedkommende Prest forgjeves har anvendt sine Formaninger ; men denne nu henved halvhunoredaarige Mand havde nok aldrig om saadanne Ting veret tiltalt af nogen Prest eller anden Christen ; han befandtes endog at vere udebt . I Tugthusets Prest fandt han en saare venlig og omhyggelig Sjelesorger ; men hvor Tugthuslivet blev ham pinagtigt alligevel ! Han maatte begynde med ABC . At lere at lese er en vanstelg Kunst allerede for Smaabern , der dog saa vidunderlig let lere at tale det kunstige Sprog ; men trefold vansteligt maa det vere for en Mand , som fra sin Ungdom af kun har ovet sig i Omstrciferlivets Ustadigheder og dertil er kommen ind i den Alder , hvor <b>Sjelen</b> allerede er mere « stikket til at lere noget Nyt . Og dertil forn , at Manden havde en betydelig udviklet- Ster paa begge Mnene ! Der kom En ind i Skolestuen og Horte ham netop stave : U-b üb , u-f uf ! og saaledes gik det med Suk og Uf den hele Tid . Han begyndte endelig med Katechismen ; men de underlige Bogstaver lagde sig som et Net for hans omtaagede Blik og hindrede ham i at se og fatte den heie Lere , som laa bag Bogstaverne ; saa tenkte han paa de mange Aar , den vanskelige <b>Lerdom</b> vel vilde tåge fra hans Frihed , og han stirrede paany med anstrengt Sands i Bogen ; men Oinene leb i Vand , og han kastede Bogen i Gulvet ; hans Kamerater trostcde og opmuntrede ham , og han forsogte atter ; men atter leb det i Et for ham , og han trampede i Gulvet og sireg heit af Fortvivlelse . Derhos maatte han tenke paa , at hans „ Kvindfolk " ( det konventionelle Navn for en <b>Hustru</b> i naturligt lEgtestab ) nu gik ude paa de vide Veie , maaste i fremmede Fanters Selstad , hun , som dog kanste i mange Aar havde bevaret ham sin Tro . Omtrent 5 Fjerdingaar vårede dette ; saa kunde han Katechismens 5 Parter tarvelig udenad , og dermed dobtes han . Han knelede for Alteret og aflagde Loftet ; han beiede sit Hoved over Fonten , og den hellige Handling fuldbyrdedes ; men ligesaalidt nu som fer sporedes der i hans Mine og Holdning noget Tegn til levende Erkjendelse

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008091603025'target=_>, 1843, Kongelige Rescripter, Resolutioner og Collegial-Breve for Norge</a><h4>1143</h4>tcrkomme , hund han i saa Mnnde e Pligtig ; men , nnar saadan Advarsel ikke maatte virke Forandring og Forbedring hos hclm , stal Saadant for Directionen forestilles , som har Net til strår , uden nogen foregaaende Opsigelse , fra Stiftelsen at udsette ham , og desuden sssal han ) efter Omstendighederne vcere videre Tiltale og Straf undergiven ; og antages derpaa af Directionen en anten redelig , duelig og bcquem Mand i den UdsatteS Sted . 8 7. Aarlig stulle Vornene i Commissionens Oververelse offentlig ernminereS , for at aflegge Prove paa den Fremgang , de have gjort saauel i <b>Lerdom</b> sem nyttige Haandgjeiiinge , og Vedemmelsen indfores i en af Lereren dertil holdcndes Protocol , for deraf at erfare den Forbedring , Vornene Tid efter anden gjore i Kundstab om det for dem Nyttige , Gavnlige ug Nodvendige at lere . 8 8. Vom ene ssulle forblive i Stiftelsen , indtil de ere fremholdte til Confirm at ion , og derefter hensettes Drengene til Haandvele , Sovcesenet eller anden nyttig Haandtering ; men <b>Pigerne</b> festes i stadig Tjeneste ; dog , om Leilighed kunde giues , iser for Drengene , for den Tid at faae dem anbragt til nyttige Haandveter at lere , efter at de haue lert det Vigtigste ca , Nodvcndigste for dem , fan det tillates , nåar Commissionen anser det at blive til Vornenes Gavn og Vedste . Paa det Pigebsrnene ikke ssulle henfidte paa deres egen Haand , efterat de ere udkomne fra Stiftelsen , sial , forinden de derfra udgaae , drages Omsorg for , at de kunne hensettes i stadig Tjeneste ; til den Gnde ssal Forstanderen ved hver saadan Leilighed baade i Intclligents-Seddelen befjendtgjore , forn og for den eedsvorne Feste- eller Stcrd-Kone anmelde , hvilke eller hvor mange som til Tjeneste udgaae , ug Krefter for dem velge de bedste og forteelagtigste . 8 9. Foruden at Vornene, i den Tid de ere under Stiftelsens Omsorg , ssulle anholdes til nyttige haantgjeninge , ssal dermed og haues Hensigt til Stiftelsens Nytte ogFor- Kel ; til den Ende ssal og tragtcs efter , at Vornene forarbeide det llldent og Linned , som til deres Kleder medgaaer , og det Fornodne hertil paa Stiftelsens Bekostning anskaffes , saavel mcd den Underuiisning , Bernene hertil beiMc, som med Indretning af Veve-Stole , og Indkjob afllld , Lim og Strie . De Born , som heri <b>vise</b> sig meest flittige og lervillige , stal gives en Kledning af det 3 o > , de selu maatte have uirlet , nåar de fra Stiftelsen udgaae , ! > I Bclonning for dem og Opmuntring for Andre . Slulde Tilvirkningen sircttke Ng uidere , end til Stiftelsens Fornodenhed udfordres , ssal det Ovrige ved Auction bortselges , Stiftelsen til Bedste . § 10. Nåar Sygdom iblandt Vornene maatte indtreffe , stal deres Cuur paa Stiftelsens Bekostning bcsorges . Hertil ssal et Verelse med de fornedne Senge stedse vere i Veredstab , at te Suge strår kunne stilles sra de friste Vom , og med en af Vyens Ale « UeiB accordees enten for en <b>vis</b> narlig Vetnling mcd Tilleg af Medicamenter , eller paa hvad anden Maade Commissionen anser fordeelogtigst til de Syges Helbredelse at foranstalte . 8 11 - Ligesom det ikke lMtviules , at jo Stiftamtmanden og Vissoppen , som de , Oue-Opsynet er « nfetroet , Vaage over , at Alting efter denne Fundation paa det noieste cver- Kldcs , saa stulle de eg uere bemyndigede , enten huer for sig eller enirjunelim, saa ofte de finde det fer godt , at lade sig underrette om Oeconomien i Stiftelsen , at late Vornene overher » i Henseende til dens Fremgang , og endelig at overlegge med Commissionen , hvorledes det havendes Oiemed til Ungdommens Underviisning og Opdragelse bedst kan Maaes , samt derefter at gjore ved en Directions-Vestutning de Bestemmelser, som de anse de norvendigste og nyttigste , saa at Intet , under hvad Navn > ne kan . sial vere dcres Bestyrelse nnddraget . Til den Ende sial bm og tilfjendegiues , hver Gang ben offentlige Eramination foregaaer . paa Kt te , om de det fornodent eragte og deres Forretninger tillade dem det , bttved kunne uere oververente .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111500059'target=_>, 1878, Illustreret Verdenshistorie</a><h4>3717</h4>Medens Kongeriget Nav arr a i dette Tidsrnm var uden Betydning , gik Portugal rast fremad . En Rcekke udmcerkede Fyrster hcevede i Lpbet af Middelalderens to sidste Aarhundreder det portugisiske Folk til 3 Sre og Berpmmelfe . Den forste af disse Konger , Dionysius den <b>vise</b> ( 1279 — 1325 ) , erhvervede sig ved sin Nidkjcerhed for Folkets Vel Tilnavnet,,Fcedrelandets Fader " . Han var en stor Ven af Kundstaber og <b>Dannelse</b> og stif- i Lissabon et Universitet , som han smere flyttede til Coimbra . Hans Spn , Alfons IV ( 1325 — 1357 ) , var en tapper Fyrste og hjalp Kongerne af Kastilien og Aragonien med at tilbagestaa Maurernes Angreb . Ogsaa af Rigets indre Styrelse indlagde Alfons sig store Fortjenester . Men en grusom Voldsgjerning klceber ved hans Navn . Hans celdste SM Dom Pedro , som var gift med en fornem kastilianst <b>Dame</b> , havde fattet en lidenskabelig Kjcrrlighed til Ines de Castro , en af de Hoffrpkener , som hans Gemalinde havde fprt med sig fra sit Hjem . Ester sin Gemalindes Dpd lod Dom Pedro sig i al Hemmelighed vie til sin Mnne Elsterinde . Men den Bevaagenhed , Dom Pedro viste Ines ' Brødre og andre Kastilianere, vakte Hofmcendenes Misundelse , og de fandt en Swtte i Portugisernes Nationalhad mod alt , hvad der var kastiliansk . Man overtalte Kong Alfons til at samtykke i Ines ' Dsd . Medens Dom Pedro var fravcerende paa lagt , indfandt nu hans Fader sig , ledsaget af nogle Hofmcend , i det Kloster i Coimbra , hvor Ines levede skjult med sine tre Bprn . Ved Synet af disse uskyldige rprtes Kongens Hjerte , og han syntes tilbpielig til at lytte til deres Moders Bm om Naade . Men da Hofmcrndene , bebreidede ham hans Svaghed , gav han efter , og Ines faldt for Mordernes Dolke ( 1355 ) . Fortvivlet over sin Hustrus Dpd , greb Dom Pedro til Vaaben , og det af Borgerkrigen hjemspgte nordre Portugals Indvaanere maatte bpde for Hofmcendenes Forbrydelse . Det lykkedes dog Dronningen at mcegle Fred mellem Fader og SM , og denne maatte med Ed love Ines ' Mordere Amnesti . Meget tydede dog paa , at Dom Pedro endnu ppnsede paa Hevn , og paa Alfons ' Raad flygtede derfor de tre Hofmcend , forn havde pvet Mordet , til Kastilien . Men efterat Dom Pedro ved Faderens Dpd 1357 selv havde besteget Thronen , traf han Overenskomst med Kongen af Kastilien, at denne fkulde udvexle Ines ' Mordere mod nogle oprprske kastilianske Adelsmcend , som havde sundet et Fristed i Portugal . En af Morderne

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011071820011'target=_>Lie, John, 1880, Staali Storli</a><h4>1477</h4>« Hvor vil du hen ? » Han spurgte med en <b>vis</b> Heftighed i <b>Maalet</b> . Det var nemlig saa , at Aslak ikke gjerne vilde vide sin <b>Kone</b> ude blandt Folk , især mellem hendes Slægtninger . Han var ræd at hun skulde fortælle hvad Mand han var . Og om han end ikke brød sig stort om hvad Folk sagde om ham , saa ærgrede det ham dog at tænke paa , at hans <b>Kone</b> skulde blive beklaget som den , der led Ondt og Uret , og var Ulykkelig . Nei , han havde Grund til at beklage sig og blive beklaget , som var bunden til en slig «Fille-Kjærring», der ikke havde Forstand paa at være om sig , og at øge sit Gods ! Men de fleste Folk ere jo saa dumme , at de vil give hende Ret og mig Uret , saa det er bedst hun bliver hjemme . Saaledes tænkte han , og da <b>Konen</b> taug stille til hans Spørsmaal , saa var det ham en velkommen Anledning til at overskjælde hende .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010092803012'target=_>Kingo, Thomas, 1858, forordnede Kirke-Psalmebog</a><h4>1568</h4>6. Slet Ingen udi Verden her Fuldkommen er at finde , Der sindes Feil hos En og Hver , Det vcere Mand og <b>Qvinde</b> ; Thi bsr vi den , som flikker sig Ei vel , men er uordentlig , Med Kjcrrlighed paaminde . 7. Den , som i Sind nedslagen er , Guds Trost vi bsr tilsige , Dg hjoelpe op med stor Begjcer Den Faldm og Skrobelige , Dg Dndt med Dndt ei lsnne saa , At vi fra Kjcerligheden gaae , Dg miste saa Guds Rige . 8. Gid Jesu Mede i os boe , Og Bsnnens <b>Aand</b> os rsre , At vi vor Gud med Glced ' og Ro Taksigelse kan gjsre , Dg aldrig staae Guds <b>Aand</b> imod , Men troe hans Drd af Hjerterod , Det elske , oere , hsre . 9. Giv os , o Gud ! din Helligaand , Hvorved vi ret kan prove Hver <b>Lcerdom</b> , og med sund Forstand

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010021803054'target=_>Birkedal, Vilhelm., 1857, Naadens Sorg og Sorgens Naade</a><h4>340</h4>imod Herren , er af den Bestaffenhed , at der ingen Grund er til at mene om ham , at han endog kun havde begyndt at stimte Evangeliet . O , Venner ! hvor fattige ere igruuden disse Tanker ! Og var Jesus end , som Han visseligen er , den storste „ Lerer , kommen fra Gud , og kunde Du end hos Ham faae Forstand paa alle Hemmeligheder i Himlen og paa Jorden , men Han var Me Mere end en saadan Lerer — da var Du ligefuldt under Fordommelsen og kom aldrig ind i Himmeriges Rige . Ak , mm er det Me de Allerflestes Udbytte af deres Omgang med Vorherre Jesus ? Jeg taler her Me om de aldeles Forvorpne og Verdenstrellene , eller om de ganske Vantro , som Me engang ville „ lere " af Jesus , men kun af sig selv eller af Verden , hvorledes de stulle bere sig ad . Men I som dog nok ville bsje Eder for Jesus , og mene at folge Ham — hvad Mere have I af Ham end dette : „ en Lerer " , en ny Lovgiver, en Moses , kommet fra Gud , for at give Eder <b>Kundskab</b> og Oplysning om den gamle Lov med en Smule Tilsetning af noget Nyt ? — Ja , Mend og <b>Qvinder</b> ! maae vi Ordets Tjenere Me , nåar vi see paa det Store og Hele af vor Virksomhet , i den ganske Menighed , maae vi da ilke tilstaae , at Summen af vor Indvirkning paa Folkene kun har veret denne : at vi have paatrykket , vedligeholdt og forplantet den <b>Bevidsthed</b> hos Massen selv i det saakaldte christne Folk , at der

<h4>1011</h4>Folk stokkes sor at holde Gilde og til Forlystelse — der Horte Du maastee , at Sandhed og Ret blev traadt under Fedder af de Ni , at levende Christendom blev haanet og Guds Born bagtalte af de Fleste — Du fornam maastee nok , at det var galt , men Du kunde itte stille Dig fra de Ni , enten taug Du stille , eller nåar Nogen spurgte Dig : „ Er Du Me ogsaa en Galilceer ? Dit <b>Maal</b> rober Dig " , ' ) — nej , det var altfor voveligt at beljende det , Du turde itte antage Dig Retfcerdighedens Sag og vende tilbage til Din Herre og tatte Ham . Hvilket Heltemod ! Og , nåar Du maastee siulde afgjore vigtige Sager med Andre , Sager , i hvilke enten Ret og Sandhed stulde sejre eller ligge under , eller under hvilke Medbrodres Vce og Vel var for og siulde forhandles — da havde Du maasiee en hemmelig Folelse af , at Du siulde ikke sporge om Andet end om , hvad Vorherre Jesus vilde dommr i disse Sager og om Din Fcerd , at Du skulde gaae til Ham og lade Hans <b>Aand</b> og Kjcerlighed oplyse Dig , at Retten kunde faae sin Gang ; — men nej — det kunde Du itte , — Du saae Dig omkring , hvad de Fleste tcenkte , hvad de , som Du meente , havde Magten og det storste Tal at raade over , havde Lyst til , og saa blev Du en „ labroder " , stilte Dig ikke fra „ de Mange " , men lod Dig binde med Trceld omslcenker af « de Mange " . O , hvilket Heltemod ! Naar Krigsluren lyder , da stokkes Stridsmcendene og styrte midt ind i Fjendernes Skare , — nåar Jesus blceser til Eiorm og Kamp — hvilket Heltemod at krybe i Skjul ! — Eller Du Son og Datter , Du Tjener og <b>Tjenerinde</b> ! — Fader og <b>Moder</b> , Huusbond og Madmoder Horte maastee til de Ni , var maastee imod - levende Christendom — saa turde Du itte bekjende Din Herre og Frelser , som bekjendte Dig for Pontius Pilatus , og nu vil bekjende Dig for den himmelske Faoer ! Hvilket Heltemod ! — Tro ikke , at Du stal gjore store Ting — stille og ydmyg stal Du vende tilbage til Din Herre . Midt i et vantro og verdsligt Huus kan den simpleste og ringeste Sjcel skille sig

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009020600117'target=_>Sagen, Lyder, 1834, Læsebog for Børn og Ungdommen</a><h4>2630</h4>Overveielse , hvilke Tanter eller Lcerdomme de mu-e ligt kunde indeholde " . " Der er maastee intet , som vev Ettergivenhet) voxer mere end en ustadig Laselyst , i Sardeleshed, nåar der gives en saadan Leilighed til at tilfredsstille den . Jeg troer , at een af Grundene , hvorfor der findes sta talrige Exempler paa <b>Lardom</b> blandt de fattigere Klasser , er den , at den fattig ? Yngling med de samme Aandstrafter er indstrantet til en snever Kreds , og nsdt til at lagge Baand paa sin Lcesctyst , og nsdvendig ret grundige maa gjennemstudere de sta Beger , han besidder , fsrend han tan anskaffe sig fiere . Edward derimod laste , liig hiin Epicuraer , som alene gad tåge en eneste Bid af en Fersken paa Sselsiden , itte et Dieblik langere i nogen Bog , efterat den havde ophsrt at pirre hans Nysgjerrighet ) eller Interesse " . " Hans Foraldre havde vistnok , ved at <b>vise</b> nogen Opmartsomhed , kunnet forebygge den Aandsadspredelse, som er uundgaaelig ved en saa overfladisk Lcesning . Men hans Moder

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112000014'target=_>, 1862, For Hjemmet</a><h4>998</h4>Dog var han i disse fjorten Dage blevet et andet Menneske og syntes efterhaanden at finde sig tilrette i Omgangen med den lille Kreds . Rigtignok var han fremdeles ofte meget forlegen ligeoverfor de unge <b>Damer</b> , og til sine Tider kunde han sidde ganffe taus i halve Timer . Sadvanligviis tog han dog artigt og temmelig belevent Dccl i Underholdningen og viiste , at der i hans Hoved var ophobet en betydelig om end vistnok temmelig uordnet Masse af Viden og de meest forskjelligartede <b>Kundskaber</b> . Og en sjelden Gang . naar han felte sig ret in « teresseret af Themaet . kom der pludselig fra ham en Rakke af fine lagttagelser . af trassende Bemaikninger . aandrige Indfald , og isar en Livlighed og — Lethed , kunde man sige . som stod i den salsomste Contraft til hans alvorlige Holdning forresten , og som man efter Friherrens Brev og eftcr Grevens egen fsrste Optiadei , allermindst skulde have ventet hos den i Eensomhed opdragne dannede Mand . Men indtraf saadanne Oicblikke nersten lynsnart , saa forsvandt de ogsaa strar ligesaa hurtigt igjen ; det var ligcsom om Greven blev forskrakkct over sin Livlighed og vilde straffe sig derfor , thi han hensank gjerne da for den svrige Dccl af Aftenen i en desto stivere Afmaalthcd og Taushed. Grevinde Charlottcs Selffabsdame havde ikke Uret . da hun engang yttrede . at Greven havde en <b>vis</b> mystisk Tiltiakningskiaft.

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013020524028'target=_>Holberg, Ludvig, 1861, Ludvig Holbergs Heltinde-historier</a><h4>2523</h4>en Dotter af den Franske Konge Nenr . 2 og 6 « tom til Verden 1552. Om hende bliver intet talet udi Historien fprend hendes Giftermaal, man merlede alleene hos hende store Sindets Gaver blandet med store Lyder . Ndi det Aar 1572 kom hun vaa den store Skueplads , da Wgteflab af Stats-Kai » on blev sluttet mellem hende og den unge Konge af Navarro , som siden regierede udi Frankriig under det Navn af Henric 4. havde ingen Lyst til dette LEgtestab , og om man stal troe , hvad hun selv haver anfsrt udi hendes Historie , da grundede denne hendes Afstye sig vaa kLljZicmtm , efterdi llLM ' io betiendte den Calvinske <b>Lcerdom</b> . Dette foreholdt hun og hendes <b>Moder</b> , da hun forestog saadant Giftermaal ; Thi , siger en <b>vis</b> endstwnt hun var heel var hun dog gandste Ortdoclox , og tilstrev man hendes 0 r - Mo6oxi6 hendes 6 ouv6lnaiit6 I > la6am6 66 duNon , hvilken havde stor Omsorg for at indprente hende Iver og 3Erbsdighed for den Romerske Kelton , og at hindre hende fra Bsgers Lcesning . Det var at pnste , at 51 a6 . 66 med famme Fliid havde indprentet hende

<h4>2791</h4>forladt : De , som med Bestandighed holdte hendes Partie , var Nasy og ovenomtalte Af dem bliver hun ophsyet over alle Mennester ; thi nu bliver hun kalden de Troendes <b>Moder</b> , nn den Catholste Verdens Lys , nu en <b>Qvinde</b> uden Synd , helligere end Jomfru Maria selv , <b>begaved</b> med hoyere <b>Kundskab</b> udi guddommelige og menneskelige Ting end Propheter og Aftostler , Christi Brud beklcedet med Solens Straaler , omtalt udi Salomons Hoysang og andre Skriftens Byger . Ikke at tale om andre Mres Titler , som bleve hende tillagde . Hun selv havde stor og hpye Tanker om sig selv , berammede sig af sine hsye Aabenbaringer , foregivende sig at have en vrovhetist <b>Aand</b> , og hendes Skrifter at vcere af den Hellig <b>Aand</b> 6 iet6l-666 , hvorfore hun og vilde , at man skulde agte dem hpyere end Skriften selv . Hvad sig anbelanger hendes andre underlige Meeninger i Religionen , da beskyldes hun for at have forkastet den Christne Troes kun6am6nt6i- , foragtet GUds Ord , og negtet Sacramenternes Fornødenhed . Men man kand sige , at det er vanskeligt at udleede af hendes Skrifter alt , hvad hun beskyldes for , tvertimod den Betiendelse, som hun 1675 overgav til ivrigheden i Slesvig , synes at befrie hende for adskillige paalagde Lcerdomme . Udi samme Betiendelse gisr hun saadan Erklcering :

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014021807176'target=_>Høyen, N.L., 1876, Niels Laurits Høyens Skrifter</a><h4>1375</h4>sig bekjendte med alle Tidens Ytringer i det 16 de og 17 de Aarhundrede , og dels ved den Lyst , der vaagnede igjennem den historiske Roman . Denne Objectivitet var en ganske anden ; den fordrede en ulige møjsommeligere Bestræbelse . Det gjaldt her ikke om at holde sig til et Par Konstnere , som man valgte efter eget Lune og egen Lyst , men om møjsommelig at samle sig <b>Kundskaber</b> igjennem Monumenter , igjennem Beskrivelser , igjennem Relikvier , som Middelalderen eller yngre Mestere lige ned til det 17 de Aarhundrede havde eftergjort , og det var ikke saa let en Sag . Altsaa maatte det nødvendig følge dermed , at selve denne Trang til at give et fuldstændigt Billede at ' en svunden Tid maatte hos det underordnede Talent udarte til en <b>vis</b> Bestræbelse for tro at copiere Klædningsstykker og Bohave eller maaske ogsaa nu og da men det var mindre sædvanligt en eller anden Bevægelse i dette eller hint Portræt , hvis man havde lagt Mærke dertil . Men for det virkelige Talent derimod , gjaldt det om , hvis det ellers var muligt , at holde sig frisk imellem alle disse Bestræbelser , og det lykkedes ogsaa en Del , trods al denne Udpyntning paa det 17 de , 16 de eller 15 de Aarhundredes <b>Moder</b> , endda at faa Sjælen til at være det Dominerende .

<h4>1692</h4>den unge , sorgfri Tambur , der midt i Kugleregnen har Tid nok til at tænke paa og tåge sig af Regimentets tro Ledsager, den lille Hund , son er bleven strejfet paa Poten , og som han derfor nu , i Selskab med sin Ven Hornblæseren , er i Færd mel at forbinde . Vi se ham som den gamle Grenader , graahaaret og afhærdet , men han kan være blod som et Barn , og det gotter ham , at den lille <b>Pige</b> paa hans Skjod ikke er bange for hans isprængte Skjæg . Afsides ligger den unge smukke Trompeter ; bfat var Døden , Tommen hviler endnu om hans Arm , urolig snuser Hesten til ham , ængstelig staar Hunden ved hans Hoved ; men Kameraterne forfølge Fjenden , de ville hævne ham , og han ligger paa Sejrens Grund , medens Soldaten paa Valpladsen ved Waterloo maa gjemme Ørnen i MHldet , som skjuler de faldne Brødre , for at den ej skal forherlige Fjendens Trofæer. Gardisten , som har fulgt sin Kejser til Elba . staar med Geværet ved Foden og Armene korslagte , mørk er Fremtiden , dog ej uden Haab ; men kun den bitre Erindrinoer tilbage for Soldaten , der i Bluse folger Ploven forbi de afbrændte Bygninger og nu opdager det Kors . som Plovjernet opkastede , idet det gik hen over en Faldens Hvilested . Det var lykkedes Horace i disse Billeder , at give det saakaldte Genremaleri et nyt Præg . Den Charakter . som her fremstilledes , har en Betydning . der gaar langt ud over den snævrere Kreds af det huslige Liv , som man var vant til at se behandlet i slige Compo ? itioner . I Stedet for det Føleri , som , efter Greuze , var bleven saa gængs , traadte her Følelse og Alvor ; i Stedet for en Efterligning , der var bundet til sin Model eller , om man vil , til sine Modeller , var her en frit skabende <b>Aand</b> . Sammenholder man denne Kreds af Forestillinger med de ovenfor anførte Emner , saa vil man overbevise sig om i hvilket Omfang dette rige Talent bevægede sig , og det Bifald , som ydedes ham i saa fuldt <b>Maal</b> . kan ikke andet end lyde som vel fortjent i vore Øren . Naar Datidens Kritik imidlertid savnede Idealitet og btil i hans Arbeider , saa sigtedes dermed ikke til de Mangler som med Rette kunde paapeges i dem , heller ikke til ■ saadanne Fejlgreb som det , han havde begaaet i den bekjendte Composition , der kaldes Napoleons Apotheose , hvor han ikke havde formaaet at bringe saa uensartede Elementer, som Vennernes Sorg paa St . Helena , de forudgangne Heltes Aabenbanng og en allegorisk Antydning af Napoleons Skjæbne , til Samklang ; nej , det var kun den evige

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081403018'target=_>, 1861, For Hjemmet</a><h4>327</h4>Foruden hine Dages fornemme Londonner Selstab gaves der ogsaa et andet , itte anerkjendt Selstab , der msdte det virkelige paa offentlige Steder og gik dandstnte , syngende , spillende og drikkende gjennem Verden , uden noget andet <b>Maal</b> end Fornsielsen . og som i Luxus , Glands og Skjsnhed overstraalete endogsaa Modens sande Konger . Da den bekjendte Miss Cunning for Exempel kom til Pris som Lady Covently og gjorde Regning paa her at mode den samme Beundring , som havde fulgt hende og Sosteren gjennem England , synes det , som om hun bragtes til at tåge Flugten for en ingenlunde hoibaren engelsk Skjsnhed , der vr endnu yndigere i Parisernes Oine . En <b>vis</b> Mrs . Pitt tog en Loge i Operaen ligeoverfor Gnvinten , og var saa meget deiligere end Hendes Herlighet » , at Parterret udraabte , at dette vr Englands sande Perle , hvorpaa Lady Coventry forlod Paris i afmagtig Vrede . Den arme <b>Dame</b> dste af Teiing , fremskyndet, sagde mun , us den hvide og rode Sminke , hvormed hun overkalkede sine saa upaastjonnete Indigheder . Samtlige

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016042908067'target=_>Schweigaard, Anton Martin, 1860, Commentar over den norske Criminallov</a><h4>3922</h4>Loven har i dette Capitels ZZ 15 og 19 gjort Straffen eller dens Skjerpelse afhengig as , at <b>Fruentimmeret</b> er under en <b>vis</b> Alder , af hvilke Lovsteder det sidste iser er af Vigtighed , deels fordi det angaaer Tilfelde , der lettere forekomme end Bortforelse , deels fordi det ved 15 Aars Alderen satte Grendsestjel her gjor Udslaget ikke mellem en stsrre og mindre Straf , men mellem Straf og Straffrihed . Skulle disse Lovsteders Bestemmelse ikke blive saa godt som uvirksomme , saa vil der til Mandspersonens Feldelse ikke kunne kreves nogen mere positiv <b>Kundskab</b> om <b>Fruentimmerets</b> Alder , end den , der kan udledes af hendes Udseende , hvilket i de fleste Tilfelde vil vere tilstrækkeligt til i det mindste at vekke hans Formodning om , at han staaer i Begreb med at handle mod Loven, hvorfor han , nåar hendes Alder senere oplyses at være under 18 , 15 eller 12 Aar , ikke kan fritages for den bestemte Straf , med mindre enten en usedvanlig physisk Udvikling gav ham en serskilt Grund til at forudscrtte en hoiere Alder , eller han paa sin Erkyndigelse om Alderen har faaet en feilagtig Underretning , ifolge hvilken det farlige Alderstrin skulde vere tilbagelagt .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009042103001'target=_>Krummacher, F.W., 1856, Adventsbogen</a><h4>525</h4>losua er renset fra Synd , domt retfcrrdig , og uagtet alle Satans Indvendinger , og cfterat denne bestemt er bleven afviist , declarerct som Ippcrstcprcst . I denne Egenskab inodtagcr han nu sin Instrnction , med dertil knyttet Forjcettelse i Tilfcelde as tro Pligtopfyldclse . Den talende er og bliver Engelen Jehova , den Hoiestes Son , men som nu ikke mere taler i sit eget Navn , men , hvilket han udtrykkelig giver at forstaae med Ordene :  Saa siger den Herre Zebaoth " , i sin himmelske Faders . I Hans Wrinde viser han da ncermest losua , hvorledes det paaligger ham , Helligdommens Tjener  at vandre paa Hans Veie og at tåge vare paa Hans Varetcegt " . Merker , forst esterat han er blcven isort Salighedens Klcrder , stilles Fordringen om en gudelig Vandel til ham ; forst , efterat hans Hjerte er blevcn trostet , ventes det as ham — thi nu forst har han faaet Evne og Munterhed dertil — at han skal  lobe paa de guddommelige Besalingers Vei " . Intetsteds i Skriften <b>moder</b> Herren os som en Eaadan , der vilde hoste , hvor han ikke forst havde saaet . Overalt bliver Hellig gjorelsen som Frugt og Folge sat ester Retfcerdiggjsrelscn , og intctstcds som Aarsag eller Betingelse foran . Overalt bliver forst Trcect plantet og pleiet , og derpå a gjort Regning paa Bladsmykke , Blomster og Prydelse; men da ogsaa med Til lid , og det med al Grund og Foie , fordi et nyt Liv ikke kan vcere tilstede , uden tillige at <b>vise</b> sig faktisk og virksomt . Men aldrig bliver den Netfcerdiggjortes Salighed gjort afhcrngig af hans nye Vandel ; den beholder tvcrtimod sin eneste Basis i Christi Netfcerdighed, i Christi Fortjeneste . Nigtignok tildeles vi ester det <b>Maal</b> af T r o sk a b , hvormed vi gaae frem ester Guds Ord ,

<h4>918</h4>paa Guds Vegne forkyndes dig , at Herren er med dig , og du en Velsignet blandt <b>Kvinderne</b> ; " men netoft dette forvirrede hende , og gjorde hende bestyrtet . Hvorledcs, mig vederfares det ? " maatte hendes forste Tanke vcere ;  mig stakkels uvcrrdige <b>Pige</b> en saadan Imodekommen af den Allerhoieste ? " Og det var hende ligesom hun maatte protestere mod saa stor Naade , og ved at asslaae den ligesom redde Guds Hellighed , som her foer vild og tog feil . Her flod , hvad der for det meste er Tilfcelde hos de Troende under lignende Omstcendigheder , <b>Aandeligt</b> og Kjodeligt sammen i bendes Folelscr . Vistnok gav Gudsbarnet , det ydmyge og sig selv opgivende , sig vel tydelig tilkjende i hendes Bestyrtelse ; men i samme Moment fremtraadte deri ogsaa det svage , skrobelige , kortsynede Menneskebarn , som i sili medfodte naturlige Indbildskheds Levning ikke betcenker , at det konuucr ud paa Eet , om meget Naade bliver det til Deel eller lid et , idet jo nctop Alt er reen , fri Naade , og den mindste guddommelige Velgjerning i samme <b>Maal</b> er ufortjent som den storste .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014061908021'target=_>Stowe, Harriet Beecher, 1876, Onkel Toms Hytte</a><h4>1148</h4>De , som handle med menneskelige Varer , bestrcebe sig saare ivrig og systematisk for at fremkalde Lystighed iblandt dem som et Middel til at drukne Eftertanken og gjore dem ufolsomme for deres Tilstand . Hele <b>Formaalet</b> for den Behandlingsmaade , der bliver Negeren tildel , fra den Tid af , da han bliver solgt paa Markedet i Norden , indtil han kommer sydpaa , gaar systematisk ud paa at gjore ham forhcerdet , tankelos og dyrisk . Slavehandleren samler sin Flok i Virginia eller Kentucky og driver den ned til et eller andet passende , sundt Sted ofte et Vadested for at fedes . Her fodres de rigelig hver Dag , og fordi Nogle ere tilbøjelige til at hcengs med Hovedet , bliver der tidt spillet paa Violin for dem , og de maa hver Dag danfe . Den , som vcegrer sig ved at vcere lystig den , i hvis Sjcel Tanken om <b>Kone</b> , Vorn og Hjemstavn er for stcerk til , at han kan vcere munter , han udpeges som tvcer og farlig , og er underkastet alle de Onder , som en i fuldeste <b>Maal</b> uansvarlig og forhcerdet Mands Nvillie medforer for ham . Lystighed , Rasthed og et fornojet Udseende , iscer naar Nogen ser paa dem , paatvinges dem altid , baade ved Haabet om derved at faa en god Herre og ved Frygten for alt det Onde , Slavehandleren kan tilfoje dem , naar de <b>vise</b> sig uafscettelige .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112300061'target=_>, 1858, Bibelen, eller Den hellige Skrift</a><h4>1416</h4>Hustru , og at det skulde antydes ham , at Kongen ikke nu trcengte til 9 hans Tjeneste . — Men Una laa for Toren til Kongens Huus med alle sin Herres Tjenere d . e . den kongelige Livvagt . Det var muligt , at Uria havde erfaret nogct om Davids Mgtcffabsbrud , som mange maatte kjende til ; dog berettes der Intet herom , men det synes som den hellige Forfatter netop derved vil lade David ftemtrcede i etvcrrre Lys , at han havde med en saa from , oprigtig og sin Herres Tjeneste 11 saa ganske hengiven Mand ai bestille . — Arken : I en saa vigtig og farlig Krig havde de medbragt Pagtesarken ligesom tilforn mod Phili-12 sterne . — Israel og luda ligge i Teltene d . e . Lsvhytter . — Bliv og idag her , saa vil jeg morgen lade dig fare : Stedse videre og dybere griber Synden om sig : Medens David ikke kan naa sit <b>Maal</b> ved at skjule den Gjerning , for hvilken Loven havde sat Dsds-13 straf , vil han nu rydde Uria afveien . — Han gjorde ham drukken , for at han paa denne Maade sknlde gjsre , hvad Listen fsr ikke havde 15 formaaet ham til . — Scetter Uria foran , der hvor Krigen er stoerkest , og vender tilbage fra ham , at han kan blive flagen og do . Han var saa <b>vis</b> paa , at denne Befaling blev udfort , at han ikke engang 18 ' behovede at ncevne , hvori Urias Forbrydelse bestod . — Ta jcndte loab hen og lod give Tavid tilkjende alle Krigens Sager . loab viser sig ved denne Anledning ligesom overalt raa og voldsom og som den , der ikke undslaar sig for nogensomhelst Gjerning ; den übetingede Lydighed , han skyldte sin Konge , kan neppe have vceret hans Bevceggrund; han maatte ane Sammenhcrngcn eller have erfaret den gjennem Rygtct , og i en for hans lave Betragtning ligegyldig Sag 21 vildc han gjsre sig ' David til Ven . — Hvo flog Abtmclcch , Icrnbbeseths I det hebraiske Sprog betyder doso ^ etli Skjaendsel ; dette var en senere Forandring af Navnet lerub Baal , der udtrykte Folkets Afsky for Afguden . Fortellingen berettes Dom . 9 , 13. — 22 - 24 — V . 22 — 24. Sendebudet tog sig Sagen meget let ; David indsaa 27 Sammenhangen meget snarere , end loab havde ment . — Ter Sorgcn var gaaen over , sendte Tavid hen og lod hente bende til sit Huus . Hvor lcrnge en Entes Sorgetid vårede , om det var over de scedvanlige syv Dage ( 1 Mos . 50 , ' 10. 1 Sam . 31 , 13 ) berettes os Intet . lalfald gjorde han Sagen af saa kort som mulig , da der i den udvortes Sorg aabenbar ikte laa nogen Anger over den begangne Forbrydelfe.

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009091403016'target=_>Grundtvig, N.F.S., 1862, Haandbog i Verdenshistorien</a><h4>1414</h4>At nu Danmark var det gamle Moderskwd for Nordens <b>Aand</b> , maae vi iblandt andet stutte af de angelsaxiske Digte , og skisndt den overveiende Dcel , Msdre sedvanlig har i Ssnncrs Storhed , er af hemmelig Art , kan man dog aldrig fortamke de store Mcrnds yngre Brsdre deri , at de crre deres <b>Moder</b> og haabe , trods Fslelsen af deres egen Svaghcd, at en saadan Moders yngste Sonner vel endnu tor ovvore til de LEldstes <b>Maal</b> . Intet Haab kan heller verre rimeligere , end at Danmark , der skicrnkede den ny Verden sin storste Digter i Angelsaxen , ogsaa vil opelske , hvad den videnskabelige Verden ei mindre savner , en tilsvarende Historieskriver, thi ingensteds aander Fwdernelandskicrrligheden endnu saa varmt , og intet Fcrderncland er saa stikket til at indgyde sine historiske Beundrere den rette Upartisthed . Skulde nemlig en Engel fra Himlen beskrive Verdens Lsb , da vilde han verre npartist , hverken fordi han fattedes Kicrrlighed til sit Fcrdcrneland , eller fordi Jordens Begivenheder var det uvedkommende , men kun fordi hans Fcrderneland var hcevet over Tidens og Lykkens Omskiftelser og sticrnkede ham paa

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111103064'target=_>Charton, Edouard, 1856, Reiser i ældre og nyere Tid</a><h4>2364</h4>deri , og til forskjellige Tider lod han en halv Snees af dem give Drikke af søvndyssende Art , saa de faldt i dyb Ssvn , hvorpaa han lod dem bringe hen i Salene i hine Bygninger . Naar de nu vaagnede her , berusedes deres Sandser af alle de skjenne Gjenstands som tidligere vare blevne dem siildrede , og oe saae sig omgivne af yndige <b>Piger</b> , der fortryllede dem med Sang og Spil og henrykkende Kjertegn . Saa fik de ogsaa kostelige Spiser og liflig Viin , indtil de , berusede af al den Lyst og Nydelse , de svwlgede i , troede sig i Paradiis og felte sig saare utilboielige til at forlade al denne Herlighed . Naar der paa denne Maadc var hengaaet fire , fem Dage , bleve de atter bragte i dyb Ssvn , og forte ud af Haven . Paa Fyrstens Sporgsmaal om hvor de i den Tid havde vceret , kunde de ei svare Andet , end :  I Paradiis , ved Deres Hoiheds Naade " , og derftaa fortalte de i Alles Paahor , de vidunderlige Ting de havde oplevet , og med Ctudsen og Nysgjerrighed lyttede man til deres Fortelling . Aloeddin henvendte sig da til dem og sagde :  Vi have vor Prophets Tilsagn , at hver den , som forsvarer sin Herre , skal komme i Paradiis , og ville I nu <b>vise</b> mig Troskab og Lydighcd , saa vil saadan Salighed vorde Eder tildeel . " Begeistrede ved s ^ ig Tale , agtede Alle sig lykkelige ved at kunde udftre hans Bud , og vare ivrige for i hans Tjeneste at lade deres Liv . Derved stete det , at naar en af ye omboende Fyrster havde paadraget sig Aloeddins Mishag , lod han ham snigmyrde ved En af dem , der paa saadan Maave vare blevne ham hengivne med Liv og <b>Sjel</b> . Ingen bcrvede tilbage for at satte sit eget Liv i Vove , thi de agtede det for Intet , og det var dem kun om at gjore at udfore deres Herres Befalinger . Saaledes blev da hans Tyranni frygteligt for alle de omliggende Egne . Tvende Udsendinge fra ham opholdt sig stadigt i Damascus og i Kurdistan , fulgte noie hans Forffrifter og opdroge Ungdommen til übetinget Lydighed . Ingen , om han end var nok saa mcegtig , kunde paa denne Maade undgaae at blive snigmyrdet , naar han havde gjort sig  den Gamle fra Bjerget " til Fjenve . Da hans Land laae i Ulaus Rige sendte denne , der havde fånet <b>Kundskab</b> om al hans Udaad , samt om hvorlunde han lod de Veifarende plyndre , i Aaret

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013042608020'target=_>Funcke, Otto, 1879, Glæde, Lidelse, Arbeide i Evighedens Lys</a><h4>302</h4>Tet var koldt Vand for mit varm ? , reiselystne Aerte ; og jeg er fast overbevist om , at dersom min Onkel havde vcrret Fader til et Dusin Sonner , havde han ikke holdt denne kolde Tale . Men da et gammelt Ord siger : „ Over de dode stal man kun holde Lovtaler," og da virkelig mange <b>vise</b> Folk hylder denne Mening , saa vil jeg idetmindste ikke ' gjore det modsatte , men efter bedste Evne tro , at den gamle Onkel har talt <b>Visdommens</b> Ord , iscer da han ' uden Tvivl havde Ret med Hensyn til Rsgningen . Imidlertid var min nbcrndige Reiselyst , som jeg . i Forbigaaende sagt , har arvet fra min kjan ' e <b>Moder</b> , skal jeg sige „ desværre " eller „ lykkeligvis " ? - - ingenlunde tanninet . Heller ikke hyldede alle Mennesker den gamle Hollcenders ovenfor ndtalte Grundsætning . Havde alle . mine Venner tcenkt som han , vilde jeg i min Studentertid itte vcerc kommet langt omkring ' i Verden . Heller ikte i Mai dromte jeq om at reise til Paris , til den store Verdensudstilling . ' Men kunde det komme mig nventet , at en af de va ' rdige Fcedre i Bremen , der er sat til at veere Præsternes Beskyttere , tom til mig og opmuntrede mig til at fslge ham og hans Familie som en Ven til Paris ? En lang Vinter , fnld af Savn og anftrcengende Arbeide, laa bag mig , og et tra ' t Hoved sad paa mine Skuldre . Jeg vilde gjerne atter arbeide ; men jeg fslte , at Legeme og Sjcrl fsrst maatte vegetere en Smule . Men det gaar ikke an , naar man bliver paa sin Arbeidsmark Kort : jeg slog til ; den gamle nbcendige Reiselyst greb mig , ug jeg pakkede min Haandknffert . Jeg sagde Hnftru og Barn Farvel , anbefalede dem i hans Hcender , som havde givet mig dem , oa. ilede til Banegaarden . Smiledes hcendte det , at jea kom til Paris .

<h4>348</h4>er Gud , „ Jerusalem , som er oventil , og som er vores alles <b>Moder</b> , " Hvad nu Paris angaar , mcerker man desvcerre meget lidet til det himmelske Jerusalems <b>Aand</b> . Den franske Hovedstad bcerer overalt tilskue , at dens <b>Maal</b> er . Forherligelsen af den store Nation . Franskmandenes bundlsse Forfængelighed , der ligesaa vel dreier sig om deres smaa Hcender og Fodder , som om Nationens Storhed , denne Forfcrngelighed , som i 1871 midt i den dybeste Elendighed dog trsstede sig med , at alle Jordklodens Folkeslag fcestede sine Dine paa Paris , og som paa en Tid , da Tystland satte sin Fod paa det sonderbrudte Frankriges Nakke , dog med uhyre Bifald modtog Vietor Hugos Vits : „ Tyskland er Folkeslagenes kjedsommelige Tipoldemoder , " denne Forfcrngelighed er verdensbersmt og er ncesten bleven til et Ordsprog . Pariserne har da ogsaa i Navnene paa sine Gaders Broer , Pladse og blotte „ foreviget " sine store Mcend og sine Seire over andre Nationer . Naar man vandrer gjennem Paris , oprulles hele Frcmkrigcs Kulturhistorie for 6 ns Grindring , men ifcrr de Seier vi ud inger , som de har tilkjcempet sig i 4 Verdensdele , Men man erindres ogsaa kun om de franske Seire ; en Katzbachplads ; en Sedanbro , en Leiftsiger, Roszbacher eller Wsrthergade har jeg intetsteds opdaget . Ogsaa alt andet minder om fransk Magt og Herlighed , og der findes meget lidet , som peger op mod Himmelen . Vistnok gives der nol af Kirker , gamle og nye , og de er tildels opforte med uhyre Luksus . Har den store Opera kostet ikke mindre end 36 Millioner Frank , saa findes der ogsaa Kirker , og det endog fra den nyere Tid , som har kostet ncesten det samme . Dette Lands Rigdom og Folkets Evne til hurtigt at arbeide sig op igjen efter alle de frygtelige Krige , Revolutioner og indre Katastrofer er beundringsvcerdig . Det er kun muligt paa Grund af den kjcempemcesfige Flid , som er en nedarvet Egenskab hos den franske Nation .

<h4>517</h4>ndftillingen ; min Forundring over denne Udstilling vcher , jo mere jeg laerer den at kjende . Kald det ikke Mangel paa Pariotisme , naar jeg tilstaar , at de kristelige Folkeslag har sprunget os forbi . Det mærkeligste er , at deres Kultur uafladelig skrider fremad Hvad leerer Franlrigcs Historie os , fom jeg nn flittig wser ? Den lagrer os dette : Da vi Kinesere for leenge stden var komne saalangt , som vi er komne den Dag idag , da rugede enduu et tykt Msrk ? over Vesten Men sammen med Kristendommen kom der en mcegtig <b>Aand</b> over disse Folk , De befinder sig i stadig Fremskridt, Men vi ftaar stille fra det ene Aartnsinde til det andet ; vor Historie er som en storartet Drypstensdannelse, det vil sige : — egentlig har vi slet ingen Historie, De knstelige Folkeslag driver det vistnok for vidt paa sin Side . De jager uden Ro og Rast efter ny Krigeråre , efter nye Love og Indretninger ; de lever stadig i en feberagtig Uro og holder nafladeliq paa med at forandre sine sociale og politiske Forhold Det er vistnok et onde ; men stillestaaende Vand raad ner ; og saadan er det hos os , vi , som udgjsr mere end en Trediedel af Menneskeslægten . Det kommer af , at vi er saa stolte og meette og indbilder os , at det ikke noget Sted er bedre end hjemme , og at det heller ikke behsver at blive bedre , end det er . ' Da jeg endnu knapt kunde tale , loene min <b>Moder</b> mig fslgende Ord : „ Vi Kinesere ser med to Dine , de kristne ser med et , alle andre Folk er blinde " . Saadan dnm og stolt <b>Visdom</b> er det netop , som gjsr os blinde . Priset vcere „ Midtpunktets Blomst , " men vi vil snart blive en Bold for Vestens Folk , dersom ikke meget bliver anderledes hos os . Jeg vilde vcerc <b>vis</b> paa dit Bifald , dersom jeg kunde gleede mig ved din Neerucerelse her . Men da Himmelen har ncrgtet mig denne Glcede , vil jeg gjsre mig Umage for i dette Brev at give dig et Indtryk af dette Lands Seeder og Tcenkeseet . Forat begynde med det mindste , saa er Franskmandenes Elskværdighed og Hoslighed meget behagelig

<h4>904</h4>Jobs Bog saa levende maler for vore Vine . O ja . vi kjender den alle , thi Goethe har jo gjentaget den i sin „ Faust ; " men hvad der af Bibelen har havt den „ 3 Ere " at optages i Faust , det finder jo ogfaa de aandrigt , som ellers anser Bibelen for en aandlss Bog . Job har mistet sin Formue , mistet sine Bsrn , mistet alt i et eneste Vieblik , dog holder han fast ved sin Fromhed , fljsnt hans egen <b>Hustru</b> spotter ham derfor . Gud gjsr Satan opmcerlsom herpaa for at <b>vise</b> ham , at der virkelig sindes fromme Mennesker , det vil sige , Mennesker , der elsker Gnd for hans egen Skyld . Da fvarer Satan med kold Haan : „ Hud for Hud , og alt det , en Mand har , giver han for sit Liv , Dog udrcet nu din Haand og ror ved hans Ben og ved hans Kjod , hvad gjcelder det , om han ikke stal bande dig i dit Ansigt , " Dette Rand er et sandt Mester stykke af diabolisk <b>Visdom</b> ! „ Mennesket , " faaledes rcrson nerer den store Forfsrer , „ Mennesket er en jammerlig Skabning , en Egoist af reneste Vand ; han tcenker kun paa at ' redde sit eget Skind Han kan opleve mange frygtelige Storme og se megen Jammer omkring sig uden at blive blod om Hjertet ; han tan endog miste sine Bsrn , men alligevel bcerer han Hovedet hsit , naar man bare ikke rsrer ved hans egetKjsd . Giv mig Magt til at sende den fromme Job legemlige Smerter , da stal du opdage , hvor gudlos den fromme Mand er , " Nu , hvad siger du , kjcere Lceser ? Ikke fandt : det manglede ikke Djcevelen Paa Menneskekundskab , paa forfærdelig Menneskekundskab , Hans srcetke Paastandc bevises ved mangfoldige Eksempler , Ak , hvilke Anfægtelser udspriuger ikke fra de ydmygende , legemlige Lidelser! hvor modlsse , hvor koldhjertede , hvor slsve bliver

<h4>934</h4>det vil sige : inderlig at bevceges over Elendigheden , saa at den bliver vor egen Elendighed , idet vi medlidende scenker vor Sjcel ned i den og af al Kraft tcenker paa , hvorledes vi stal lindre den og tage den bort . Den , som vil fatte dette , han maa drage om med Jesus , maa fe og hore , hvorledes han grceder , troster , hjcelpcr og frelser . Naarfomhelst vi fer ham , ser vi ham omgivet af en Sky af elendige ; og denne Sky af elendige , denne Sky af Vidner om Dsdens Herredomme i Verden forvandler sig under hans forbarmede Kjcerligheds Tryllestav til en Sky af Vidner om hans Herlighed. Vistnok vil der aldrig i denne Dsdens Verden sindes noget Menneske , fsdt af en <b>Kvinde</b> , i hvis Hjerte denne fuldkomne Forbarmelfe , denne hellige Mildhed, denne himmelste <b>Visdom</b> og Taalmodighed , denne guddommelige Medlidenhed bor i faadan Fylde som i hans Hjerte . Men alligevel har lefus testamenteret dette sit Sind til sine sande Diseiple paa Jorden som den Kraft , hvorved de stal forkynde hans Navn i Verden, og hvorved de stal overvinde Verden . Ligesom han er kommen for at tjene , saaledes stal ogfaa hans Difciple fremstille sig som Menneskeslægtens Tjenere . Men navnlig gjcelder det at tjene der , hvor det mest trcenges ; derfor staar de elendige , de syge , de lidende i fsrste Rcekke blandt dem , der er lagt en lesn Efterfslger paa Hjerte . Barmhjertigheden vil indtil Verdens Ende vcere et Kjendetegn paa en cegte lesn Discipel ; derved beviser han , at han har feet lefus ind i Hjertet , at han vandrer i hans Fodspor , at han er fsdt paany af Herrens <b>Aand</b> , at han eier den Hellig Aands Ild . Men hermed er det allerede fagt , at det barmhjertige Sind og det praktiske Arbeide i Barmhjertighedens Tjeneste ikke er noget , som gaar ved Siden af Troen og er uafhcengig af denne . Dette Sind er ikke en Lov , ikke en Pligt , men det er det umiddelbare Gjenskin , den umiddelbare Udstraaling af det nye

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016030808031'target=_>Pavels, Claus, 1867, Claus Pavels's Dagbogs-Optegnelser 1815-1816</a><h4>3511</h4>mere herneden . Min i fiere Henseender agt- og elstværdige Svigerfader , der vel var KjMmand som de andre , uden betydelig Aandødannelse , men hcevede sig dog ved egen Kraft over mere end een af Standens Fordomme , havde en Amhed i Hjertet , en Fiinhed og Delicatesse i Følelsen , der savnes hos mange langt mere cultiverede Kjobmnnd , og fremfor alt elskede mig og min <b>Kone</b> hjerteligene den ? 9 aarige Ungling Viskop Vrun , hvis udmcerkede Talenter og varme Iver for Religionen , og hvad han ellers fandt godt og hcederligt , give ham billig Adkomst til Samtids og Eftertids beundrende HMg « telse , om han end ikke var den feilfrie , urokkelige , standhaftige Mand , Hersleb og flere ville ham til : den HMcerværdige Valle , til hvem jeg bandtes med end kjærere og helligere Vaand , hvis Aandsgaver vel vare mindre eminente end VrunZ , men som i <b>Kundskaber</b> langt overgik denne og i Hjertets Varme idetmindste var hans Lige , der med et Mod og en Fasthed , Vrun aldrig fik Leilighed til at give Prøver paa , stod næsten ene i den svære Kamp , hvor evangelist Christendoms Fiender overmodige brammede af deres Seier og dens faa Venner droge sig tause tilbage : den i Sandhed hcederlige Knudsen , der vift havde sine Feil ( det havde de alle , og hvo har ikke det ? ) , men ogsaa ypperlige Egenskaber , som faa af dem , der dadlede og spottede ham , besad — disse sire Hcedersmcend — for mig vare de saagodtsom men med dem alle har jeg ftaaet i meget interessante Forhold , af dem alle var jeg agtet og elstet , og alle gik de med Mre i dereS Grav , med rolig <b>Bevidsthed</b> om at have virket i deres Kald og stiftet Gavn — alle ere de nu vift salige hos Gud . Held dem og Enhver , der levede som de , som de ! — At jeg endnu ikke har kunnet ncevne min gode gamle Frue Dyhring blandt de Hjemkaldte , er tungt . Jeg var hos hende i Efter » middag og tog Afsked med hende for det gamle Aar . Ak ! hun er en saare trcet Vandrer , og endnu vil ikke DFdenS

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009072703024'target=_>Hersleb, Peder, 1863, Otte Prædikener over vor Frelsers bitre Gang til Golgata samt over hans syv Ord paa Korset</a><h4>242</h4>Alting , det er , ladet Alting staa og fare , og befattcde sig ikke med nogen Ting . Men nu for at forsorge Jesu <b>Moder</b> , saa tog han sin Eiendom i Brug . Niceforus siger , at han solgte sit Gods i Galilcea og kjobte et Hus ved Jerusalem , hvor Jesu <b>Moder</b> blev hos ham indtil sin De < d , og da forlod han Alting igjen og gik til Asien . Men Nicefori Ord ere tidt befundne uvaalioelige , og derfor er dette ikke at stole paa . Nu , det var det andet Jesu Ord paa Korset , hvilket synes kun at angaa det Timelige , en timelig Forsorg for sine timelige Venner , og derfor kunde man tcenke og sporge : Har da Jesus ikke vigtigere Ting at tcenke paa paa Korset i sin Dødsstund , end at bekymre sig om saa ringe Ting ? Vor hovmodige og blinde Fornuft fynes altid at finoe Noget i lefu Ord at kritisere over . Men Troms ovlyste Vine finde ogsaa i dette Jesu Ord mange Dybheder as Guds <b>Visdom</b> . Saa ringe disse Ord synes at vcere , saa rige ere de paa <b>Lcerdom</b> og Trgst . Fsrst paa Llcrdom . Der ere utallige Ting , vi kunne lcere af disse Ord . Af Maria kunne vi lcere at elske Jesus , samt at hendcs Kjcrrlighed var stcerkere end Dstden ; thi ingen Fare , ingen Svot kunde afholde hende fra Jesu Kors . Af hende kunne vi lcere Trofasthed , Bestandighed og at vccre tro indtil Dsden , saa han kan give os Livets Krone . ( Aab . 2 , 10. ) Ia vi kunne af hende lcere at lide for Kristi Skyld med et uforsagt Mod ! Af Jesus lcere vi den Lydighcd og Kjcerlighcd et Varn , en SM bsr <b>vise</b> og have til sine Forceldre . Ligcsaalidt som en <b>Moder</b> kan forglemme sit diende Barn , ligesaalidt bpr et Barn forglemme sine lidende Forceldre . Paulus siger , at de , som elske og lyde sine Forceldre , betale sine Forceldre igjen , og det , siger han , er vel gjort og behageligt for Gud , og at den , forn ikke gjpr dette , har forncegtet Troen og er vccrre end en Vantro . ( 1 Tim . 5 , 6. ) Thi faaledes ere de Ord at forståa :  Dersom Nogen ikke har Omsorg for sine Egne , " hvilke Ord saa skammelig misbruges som et Paastud eller Undskylduing , naar man vil skrabe Gods sammen ; ja , da siger man : Paulus har jo sagt : hvo , som ikke har Omsorg for sine Egne , er vcerre end en Vantro . Men Paulus har aldrig talt om , aldrig tceukt paa den Omsorg , at Forceldre skulle skrabe sammen til Vernene , men han taler om Vernenes Pligt at forsprge deres Forceldre og Oldefcedre , at de ikke skulde blive Menigheden til Vyrde , saa lcenge Barnene nogeulunde kunde forsørge dem , og det kaldcr han

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009091403018'target=_>Goldschmidt, M., 1863, Fortællinger og Skildringer</a><h4>970</h4>holdt en Vogn , i hvilken en Mand og en <b>Kone</b> , Bpnderfolk , syntes meget beskjeftigede . Jeg var ifcerd med at tcende en ny Cigar ved Stumpen af den gamle , og det tog nogen Tid , saa at Majoren, da jeg kom tilstede , allerede var i fnld Samtale eller rettere Enetale , fordi han , sandt at sige , havde spurgt om Meget , men ikke faaet mere Svar , end om han havde talt Grcest . Sagen var , at skjondt fpdt paa Landet havde Hr. Lind , medens han bibragte den ene Generation efter den anden <b>Kundskab</b> om Mcend og Folkeslag, ganske tabt Evnen til at samtale med sine egne Landsmcend af Bondestanden , og da samtidig en <b>Bevidsthed</b> om denne Sandhed begyndte at dcemre for ham , bad han mig om at overtage Samtalen , hvilket jeg gjorde .

<h4>1136</h4>thi Kammerraad , Landvesenscommissair Moes , som sad paa hendes anden Side , gjorde egenlig Tjeneste som Vceg , og han var saa beskjeftiget med Mad og Drikke , at han knap , som mange andre Vcegge , havde Yren — faa fylte jeg dog , at Situationen ikke var ganske uden Fare , nemlig for mig selv . Det er ncesten tantalisk at sidde hos en saadan smuk <b>Pige</b> , der ikke blot er elskverdig, men betragter En som Stedfortreder for en Anden , imod hvem hun vil vere specielt elskverdig, og saa hele Tiden passe paa sig selv og verge sig imod det Behag , hun vekker . Der var Meblikke , da jeg hartad pnskede , at hun maatte <b>vise</b> sig lidt keitet eller sige noget Skurrende , for at jeg kunde tryste mig med , at det ikke var mig , men Richard eller Kammerjunkeren , der skulde have hende . Men hun vilde ikke , eller rettere , hun kunde ikke . Det var netop det Fortryllende ved hende , at man fornam en Natur , der uden overvettes <b>Dannelse</b> var harmonisk og gik rigtig ved Hjelp af sit Instinct .

<h4>2637</h4>blev Gjenstand for lidcnskabelig Concurrence isaer af engelstc <b>Damer</b> , blev Mdm . 66 S . trykket lidt tilside eller veg frivillig . — Hun har vceret to Gange gift og gjsr ingen Hemmelighet ) af , at hun blev stilt fra sin stdste Gemal , saa at jeg uhindret kan berpre denne Omstcendighed . Om han var Baron , ueed jeg ikke , og det har ikke tjent mig til Oplysning , at hun i Selskabet bliver kaldt Urne . Ili Lllroiin6 . Thi formuende <b>Damer</b> , man ikke veed Andet om , fsre i Rom fcedvanlig en saadan Titel . Mdm . 66 S . , der baade er rig og <b>aandfuld</b> , bliver undertiden af <b>Damer</b> , der ikke kjende hende og stutte fra hendes Felttog , anseet for emanciperet ; men i Omgang med hende mcerkes Intet dertil . Hun forer et gjcestfrit Hus , hvor hun hver Vinter famler det Meste af , hvad Romer - saisonen frembyder af Interessant og <b>Dannet</b> , og dcriblandt ogsaa endel <b>Damer</b> . At hun , Garibaldis Veninde , faaer Lov til at boe i Rom , kan synes paafaldende ; men Pavens Regering scetter virkelig en Stolthed i at kunne paastaae , at den ikke ncegter Nogensomhelst Gjcestfrihed , saalcenge Gjcesten ikke paa nogen Maade optrceder imod den . Mdm . 66 S . er imidlertid Gjenstand for et meget omhyggeligt , sijyndt stjult Tilfyn og maa

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111401060'target=_>, 1877, Illustreret Verdenshistorie</a><h4>1851</h4>Medcns Pompejus seierrigt forte de romerske Hcerc ligetil Kaukasus og Syriens ørkener , havde i selve Rom Staten vceret ndsat for den yderste Fare . Demokratiet var nu kommet til Magten , iscer da Nobiliteten tun havde svage Forkjempere . En af de merkeligste blandt disse var den unge Marcus Porcius Cato , en Mand af übesmittet Nedelighed og fuldstcendig gjennemtrcengt af den stoiske Filosofi , hvis Scetninger han med Strcenghed s ^ gte at gjennemftre i Livet , uden altid at tåge de givne Forholde i Betragtning. Med fit Svcermeri for den gode ganile Tid , som nu ikke mere kunde kaldes tilbage , da Forudscetningerne dertil manglede , var han kun lidet stikket til at forståa Fremtidens Krav og beherske den urolige romerske Verden . Demokratiet havde derimod en Leder af helt andre Egenskaber i Cajus Julius Ccesar ( fpdt 102 eller 100 ) , en af de rigest begavede Mcend , Historien kjender . Han forenede en usedvanlig praktisk Forstand med det cegte Genis dybe Fremtidsblik , og hans naturlige Evner vare blevne udviklede ved den alsidigste og fineste <b>Dannelse</b> . Han saa klart alle detSamfunds Ntangler , som han tilhørte , og ansaa sig , med fuld Bevidfthed om sin Overlegenhed , selv kaldet til at gjenfpde det . Allerede tidlig havde Ccesar henledet Opmcerksomheden paa sig , da han vcegrede sig ved paa Sullas Bud at forskyde sin <b>Hustru</b> Cornelia , en Datter af Cinna . Kun med Vanskelighed fik han paa sine Venners ForbM Naade hos Dictatoren , der fandt at der i den  uoptugtede Dreng " fkjulte fig mere end en Marius . Ccesar havde derpaa deltaget i Krige paa forskjellige Kanter og havde i Rom vundet et Navn ved sin Anklage mod en <b>vis</b> Dolabella og ved sin vindende Personlighed og sin Gavmildhed erhvervet sig Mcengdens Hengivenhed . Som Wdil i Aaret 65 anstillede han de mest glimrende Skuespil , ved hvilke endog de vilde Dyrs Bur vare af massivt Splv . Alt dette kosteoe store Pengesummer , som maatte reises ved Laan , og saaledes kom den dybt forgjceldede Ccesar ganske naturligt i Berøring med den rige Crassus . Disse Mcend havde forpvrigt ogsaa andre Grunde til at holde sammen mod Pompejus , hvis Tilbagekomst i Svidsen for en stor og seierrig Hcer impdesaaes

<h4>2033</h4>Denne var almindelig afholdt og agtet for sit Retsind og sine strcrnge moralske Grundscetninger . Han havde dyrket filosofiske Studier og gjort det til sit <b>Maal</b> i Livet at lcve ester den stoiste Filosofis Lcere . Da han var gift med Porcia , en Datter as Cato , kunde han paa en <b>vis</b> Maade ansees som Arving af dennes republikanske Standpunkt , og den ideale Retning, som hans videnskabelige Sysler havde ' givet ham , forklarer let , at han kunde svcerme for den gamle Forfatning uden at indse , at den nu var utidsmcessig , ja ligefrem umulig . Brutus havde fulgt Pompejus , men var dog bleven behandlet med største Godhed af Ccesar , som tidligere havde elsket hans <b>Moder</b> Servilia og nu ncesten modtog ham som en SM . Han havde dog ikke kunnet forlige sig med Tanken om Republikens Fald , og

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012060608208'target=_>Paludan, Julius, 1878, Om Holbergs Niels Klim, med særligt Hensyn til tidligere Satirer i Form af opdigtede og vidunderlige Reiser</a><h4>168</h4>Cyrano de Bergerac . De Salige maa kærape med de løsslupne Fordømte , li <b>vis</b> øde Klipp eøer ere fulde af Flammer , Svovllugt, Smuds- og Blodfloder og frembringe Sværd og Dolke i Steelet for Blomster . Overensstemmende med Lucians <b>Formaal</b> fremhæves særlige Pinsler for løgnagtige Historieskrivere som Herodot og Ktesias . Deretter passeres Drømmenes mørke 0 , der viger tilbage for de Reisende ; den allegoriske Beskrivelse af Staden med Porte af Horn og Elfenben er lige efter Odysseen XIX 560. Øen er raid af Valmuer og Flagermus ; Drømmene vandre om og føre de Reisende tilbage liver til sit Hjem . Et Sted i Havet begynder Gaasen paa Skibets Gallion at slaa med Vingerne , den skaldede Styrmand faar Håar , og Mastetræet spirer , ganske som i Kong Ha dings Reise til de Dødes og de Levendes Land i den nordiske Sagnhistorie * ) , der minder om flere af de Digtninger , vi her beskjæftige os med. — Resten af Lucians Eventyr , der bestandig faa en mere kaad , eventyrlig Character , vedkomme os mindre : saaledes de * / a Stadie høie Mennesker med Flammeglorie , der paa seilende Øer kæmpe med uhyre Polyper, Korkfødderne , der løbe paa Våndet , Sørøverne , der seile i Græskar eller ride paa Delphiner , Menneskeæderne med Oxehoved , Rygsvømmerne , de vilde <b>Kvinder</b> med Æselsfødder , fra hvilke Forfatteren med en anden Odysseus ' s Snildhed redder sine Ledsagere , Skoven , som de seile over , den 1000 Stadier dybe Kløft i Havet , over hvilken en Vandbro fører . Sluttelig strande de i en anden Verdensdel „ lige over for deres egen " , saa man skulde tro , at han tænkte sig Jorden rund og omseilet .

<h4>897</h4>I N . Kl . træder Fornuften og den naturlige Religion saa stærkt frem 1 ) , at havde Holb . ikke senere skrevet sine Epistler, hvor han „ da han i Henseende til sin Alder og sin Svaghed anser sig som en reisefærdigMand " 2 ) , bliver alt mere"orthodox, kunde man gjerne antage ham seiv for Theist , og heri kunde maaske søges et Bevis for , at N . Kl . maa være skreven paa et tidligere Tidspunkt i Holbergs Liv 3 ) . Hvorvel han erklærer , blot at ville referere den potuanske Religion uden at dømme derom 4 ) , billiger han den dog aabenbart sener 5 ) , og den korte Beklagelse af , at N . Kl . i Qvama ikke har anvendt sin Magt til at indføre Christendommen , staar rent isoleret 6 ) som en af Forsigtigheden af tvungen Indrømmelse, ligesom ofte i Utopierne . Rigtignok er den potuanske Religion ingen blot naturlig , men en aabenbaret Religion 7 ) , og at menneskelige Love og den naturlige Moral ikke er nok , vises ligesom hos Morus endvidere i Mikolak og især i det frie Land 8 ) , med udtrykkelig Polemik mod Bay le 9 ) , der hævder, at et Samfund meget vel kan beståa uden Religion . Men at denne aabenbarede Religion skulde være Christendom eller som utilstrækkelig <b>vise</b> ud over sig seiv , spores intet Steds , og den kommer nærmest til af Hensigtsmæssighedshensyn 10 ) , som Grundlag for Moralen . Meget rigtig henviser Smith 11 ) til <b>Mor</b> . T . 14 , hvor det netop om N . Kl . hedder : „ Der vises udi Skriftet , hvor meget et Folk alene af Naturens Lys og de skabte Tings Betragtning kan forfremmes udi Guds <b>Kundskab</b> . . . Autor har desforuden tåget den Præcaution at <b>vise</b> Revelationens Nødvendighed for at igjendrive dem , som udi vor Tid have nægtet dens Fornødenhed . " Tidens

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010021803034'target=_>Hansen, Maurits, 1855, Godmand, eller Den norske Børneven</a><h4>2292</h4>Berommelser . Tvertimod begyndte han nu forst ret at indse , hvor lidt han vidste , og mcd denne Erkjendelse blev Dnsiet om at lcrre Noget tilgavns levende hos cham . Nu var den gode Dreng tillige meget bcsteden og frygtsom. Derfor stod han lcrnge i Beraad , om han skulde vove et Skridt , hvorved han alene kunde tro at komme sit <b>Maal</b> ncrrmere . Dette bestod i at henvende sig til Prcrstcn og bede ham om Undervisning . Det vilde ikke nytte ham at tale mcd sine Forcrldre herom . Altsaa tog han Mod til sig og gik en Søndags Eftermiddag til Prcrstegaarden med sine Skriveboger . Rodmendc , fremførte han sit Andragende for Prcrsten . Denne gode , forstandige Mand horte mcd Venlighcd paa ham , gjorde ham adskillige Sporgsmaale , bcsaa hans Arbeide - og forundredes over Drengcns <b>Dannelse</b> . Presten lovede derpaa med megcn Fornyelse at tåge fig mere af Kristopher, og da han troede at den Oplysning , han gav ham , ogsaa vilde verre flere unge Mennesker til Nytte , tilbod han sig at give de Drenge og <b>Piger</b> , som vilde komme et Par Timer om Ugen til ham , Undervisning om Naturen , d . e . om Verdens Bygning , om Planeterne, om Jordens Overstade , om de saakaldte Elementer og om Alt , hvad der voxer i og paa Jorden , og tillige <b>vise</b> dem , hvad Nytte et Menneske har af at vide Noget om disse Ting . Hvad synes eder om dette Prestens Tilbud ?

<h4>2610</h4>Jeg maatte se surt eller blidt , saa gav man mig Deig under Hcrndernc . Du siulde lMt , hvorledes der blev snakket og fortalt under Arbeidet , nu om en syg Bondekone, nu om Onkels Havreager , og Priscrne paa Mel og Flest , om den gamle Klokkcrs smukke Sang , og jeg ved ikke Alt . Sent kom vi i Seng , og neppe var det Morgen , forent » hele Huset var i Oprstr . Klokken syv sådde vi ved Kaffebordet , som var bckrandsct . Onkel omfavnedc sin gamle <b>Kone</b> og dnsiede hende med en hjertcbrcrkkcnde Tale mange sunde Aar . Tante grcrd , og min <b>Moder</b> , der sinder Alting fortreffelig her , var rt > rt . Men mig forekom Scenen altfor mager til en Scene . Forwringcrne varc underlige . Fatter forcrrcde hende riktignok ct net Hovedtal ; men ' min gode Kusine havde et stukket Uldsijort til sin <b>Moder</b> , og gamle Grandtante en Nok , Fcrtter Anton en Garnvinde , og Husholdersken en Flddcsie . Nu siulde vi ordne vor Pynt og dcrkke Bordet , da flere Gjester siulde medtages fra Kirken. Min Pynt blev gjcnncmgaact ; den hvide Morgenkjole var upassende at gaa i Kirke med o . s . v. Endelig satte vi os i Anmarsch for at gaa i Kirke . Ester en afsiyelig Sang opvartcde Onkel med en gyselig lang Preken , hvori der ikke var en Smule Dogmatik eller Mystik , men bare gammeldags triviel Moral . Saa blev der siraalet igjen og messet og siraalct . Paa Hjemveien spadserede endel af Gjesterne med os til Prcestegaarden. Jeg havde den 3 Erc at ledsages af en ung Officier, som underholdt mig med Lovtaler over Lafontaines Familiehistorier , formodentlig for at <b>vise</b> sin <b>Dannelse</b> . Ved Spisebordet talte man om Konfirmander , Komposter, og Potetcsforedling vcd Fro . Jeg har nok vidst , at Potetcs setter Blomster , mcn at de have Fro , drømte jeg ikke om og spurgte derfor i min Enfoldighet » , om Froec var det Samme som Oinenc eller Spircrne .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010022412001'target=_>, 1845, Kongelige Rescripter, Resolutioner og Collegial-Breve for Norge</a><h4>6917</h4>Commission som Inspectorer det noiagtigste Tilsyn . De Regler , hvoreflci den bliver at bestyre , ere selgende : 1 ) Forsergelse ved Lcegd in nnturi » l , ^ : vises ikke Andre , end Bern over 9 Aars Alder , der nu kunne gjere Tjeneste som Gjeterdreng eller Barnevige , og Saadanne , der as Alderdom ikke ganske kunne forserge sig selv . men dog endnu have Krcrfter tilovers til nogen N ^ tl , i Livet . 2 ) Naar altsaa det forhen borttingede Barn har fyldt sitere A « , overscettes det i 4 de ( slasse eller blandt Lcrgdslemmerne . Dog kan og b ^ heri ssee Undtagelser ester Forssjelligheden as Barnets saavel Helbred som Legems-Sterrelse. saaledes at det baade tidligere lan udtages as Iste blasse , og vel ogsaa lcengere forblive der . Ligeledes tages det Penge-Beleb i Betrcigtning, for hvilket Barnet er borttinget , saaledes , at naar det hos retssafn . Folk , der ligesaa omhyggeligen , som dette vilde ssee hos Lcrgdsbonden , seige saa vel for dets Legems Pleie som <b>Sjels</b> <b>Dannelse</b> , end at lade det nytte Bygdens almindelige Skole-Underviisning , kan forblive til mindre Udgift svi Cassen , end ved at anvise det Lcrgd , saa forbliver det saa Icrnge i iste blasse ' og serger da ( Zom , i Forening med Districtets Inspectgr omhyggeligen for , at det i it ) , er i lAar ell . hsiere og lavere Alder , dog altid under i 5 Aar , da det borconfirmeres og forssrge sig selv , kan faaeTjeneste-Plads hos brave Folk , saa det kan udgaac af ( sasfen . 3 ) Naar derimod den celdre Fattige , der er anviist Lcrgd in nntur , , ved tiltagende Alderdom eller Skrsbelighed bliver saa ussel , at han slet ikte kan gjere Nytte hos Lcrgdsbonden , men tvertimod forvolder ham alt for stor Byrde , hvilket Inspecteren skjenner og foredrager Commissionen , da tages han ud af Lcegdet og overflyttes i Iste blasse eller blandt de Borttinget , 4 ) Ved de Fattiges Anviisning til Lcrgd in natur » tages deres Alder og Helbred i Betragtning , saaledes at de ikke altid faae fuld Gaard , men overeensstemmende med deres Krcrfter forlcrgges paa I eller Gaarde , og ved tiltagende Alder og Krcrfters Svcrkkelse omlcrgges siden paa fuld Gaard , indtil de gaae gansse ud af Lcegds- ( slassen og over ide Vorttingedes . 5 ) Da tet er befundet , at adskillige enkelte Gamle , Mcrnd saavelsom <b>Qvinder</b> , helle » modtage et mindre Bidrag i Penge end den Foregelse , man kunde anvise dem ved Lcegd in natur » , saa kan vel ( Zommissionen tilstaae dem saadant , men under strengeste Forvligtelse at ikke betle , og neiagtig at opgive for Districtets Inspecter , hvor og hvorved de have deres fornodne Ovhold . Naar de derimod ikke lcengere kunne lade sig neie med bemeldte mindre Bidrag as ( Tassen , men onsse Lcrgd in natur » , da melde de sig derom betimeligen bos Inspectoren inden Omlcegnings- Commissionen , der indtrceffer i Juni og T > e « cember Maaneder , vaa det denne , der ber have en Fortegnelse over alle til Districtet henherende Gaarde , med hosfeiet Ovlysning , enten de ere belagte eller ledige , kan opgive for Commissionen den Gaard i Districtet , som han kjender passende til den Ansegendes Helbreds- Forfatning , eller , i Mangel af ledige Gaarde i eget District , fordre en saadan ietaf de andre . 6 ) Bern forblive ikke lcengere under offentlig Forsørgelse , altsaa ikke heller som Lcegd paa nogen Gaard , end indtil de ere confirmerede ; og da dette , efter beherig Skolegang , kan og bor siee i den sidste Halvdeel af deres 15 de Aar , saa ferger saavel Lcegdsbonden , som Inspecteren , i Forening med Commissionen , i dette sidsteLcrgdeaar omhyggelige » for , at faae dem udstcedede i Tjeneste sirar efter Confirmationen . Til Erstatning for de Confirmations- og andre Klceder, Lcrgdsbonden har bekostet , og som ved Udtrcedelsen af Lcrgdet tilhe « Barnet , bliver hans Gaard fri for Bidrag indtil ncrste Hommission ; huorimod han dog ogsaa i samme Tid beholder Barnet , saafremt det ikke er lykkets at ssaffe det Tjeneste . 7 ) Naar nogen Fattig , cg fornemmelige » Bern , bliver anviist Lcrgd paa fiere Smaagaarde sammentagne , saaledes , at de , eftel en <b>vis</b> Tids Ophold paa hver iscrr , ssulle stytte fra den ene til den anden , da-ftrceber Commiss . heller at faae dem forblivende paa den ene af saadanne Gaarde , imod at tilstaae denne de andres Contingent i Penge . Dette Beleb udbetaleS da af ( Zassen , og de Gaarde , der siulde have havt Lcegdslemmet til Forsørgelse in natur » , indbetale igjen til Cassen . 8 ) Naar den Fattige er

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081903015'target=_>Sundt, Eilert, 1859, Fortsat Beretning om Fantefolket</a><h4>224</h4>. og den Forestilling paatrcenger sig , at om vi havde nsiagtig <b>Kundskab</b> om Tilstanden hos alle Fantefamilicr , skulde vi finde et ganske forfcerdeligt <b>Maal</b> af Dodelighed og Elendighed blandt Bornenc . Rimeligvis i langt storre Grad end hos den bosatte Befolkning viser denne Elendighed sig ogsaa deri , at Bornene blive dovstumme ( Sagerne No . 51 , 67 , 37 ) , vanvittige ( No . 76 , den der m ; der No . 18 — 23 ncevnte Familie ) , eller yderlig sleve paa baade <b>Sjel</b> og Legeme ( No . 45 ) ; " ) Ercmpler paa <b>Piger</b> med den faldende Suge ' forekommer under Sagerne No . 61 og 44. Og nu bevirker vel Bornenes Dsd i de fleste Tilfcelde ^ ) at saadanne store Skrobeligheder og Lidelser ikke faa hcenge med til voren Alder ; men meget ofte berettes det tillige om de Vorne , at de ere i hoi Grad lidende og kncekkede og det formedelst vildt og uordentligt Liv i deres raste Ungdoms-Aar . Kort , Tilstanden er mangengang saadan , at man fast kunde kalde Understottelses- Pengene vel anvendte , om Pcrsonerne kun holdt sig i Ro , til de vare komne lidt til Forlighed igjen , og i mere alvorlige Tilfcelde, f . Er . naar det gjcelder at faa dsvstumme Born anbragte ved de for saadanne indrettede Undervisnings-Anstalter , maa det jo kaldes en sand Velgjerning , at der nu håves Penge dertil .

<h4>1083</h4>tilsammen 33. De 4 Forccldre-Par levede i Konkubinat , saasom 7 af de 8 Personer vare ukonfirmerede ; alle de scelstilt ncevnte Gutter og <b>Piger</b> vare ogsaa uko n finner ed e ; der var altsaa 20 Konfirmander, af hvilke de Fleste neppe kunde lcese i en Bog . De 6 siVst optagne af disse Konfirmander ( deriblandt 1 Forceldrc-Par ) ere endnu besijcestigede med Undervisningen ; 1 Kozchrmand ( No . 24 ) koni ved en , som det synes , ham utilregnelig eller dog undstyldende Omstcendighed ttl at afbryde Konfirmalions-Beredelsen . Af de ovrige 13 blev af Prcesten Knudsen 2 debte og 11 konfirmerede, og derefter bleve 3 af hine Forceldre-Par cegteviede . Bcstrcebclscrne for at naa det ncermest foresatte <b>Maal</b> maa altsaa siges at viere lykkedes meget godt ( jeg siatcr her naturligvis til den Stadighed , Konfirmaitderne have " udviist , den Kundstab , de have erhvervtt , ikke ocrimod til den Indflydelst , Ordet har havt paa deres Hjerte ; thi den kjender alene Gud ) . Efter Konfirmationcn og Mgtcvielsen staar Sagen saa : Det har endnu ikke lykkedes at overvinde de mange Vansielighedcr med at faa de tre cegteviede Par fast bosatte , men der arbeides fremdeles til dette <b>Maal</b> , og det lader til at vcere Vedkommendes alvorlige Dnste aldrig niere at komme paa Fantesticn. Af de ovrige Konsirmander har 1 <b>Pige</b> og 2 Gutter nylig qjennemgaaet Haandvcerkslcere og stikket sia / v el ' og 1 <b>Pige</b> tmed Baril ) var nylig bleven sat i Vceverlcere ; ogsaa om disse ter det haabes , at de have forvundet bysten til det gamle Reiseliv ; men det har jo endnu ikke viist sig , om de fra nu af kunne erncere sig selv paa ordentlig <b>Vis</b> . 1 af Guttcrne og 1 af Pigcrne, som efter Konfirmatwnen bleve gifte og saa satte i Lcere , romte efter nogle Ugers Forlob og ere ikke komne tilbago , og 1 Gut og 1 <b>Pige</b> romte strar efter Konfirmationen . Af de 13 konsirmerede Personer ere altsaa 4 vendte ' tilbage til Fantestien .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092403013'target=_>Lyng, Ge. Vilh., 1866, Hedenskabets Levnetsløb</a><h4>248</h4>Men Semiterne bleve ikke staaende ved Livets Kilde og Ophav , de styrtede sig med hele deres gledende <b>Sjel</b> og hver deres dybe Aands Tanke ud i dets rivende Strem og lode sig af den blindt fere hen mod dens <b>Maal</b> . Livet er Formelingen , Enheden af det Modstridende , af Lys og Merke , af det Mandlige og det <b>Kvindelige</b> ; men disse ophere ikke at stride mod hinanden , og derfor er Livet delt mellem stridige Magter ; det faar sin Charakter derved , at Gleden uafladelig forjager Smerten , og Smerten Gleden ; det staar vexelvis under Indstydelse af Gledens og Smertens, af Livets og Dedens Magter . Guddomsmagten , der giver Livet , giver ogsaa Deden ; Solen , der ferst med sine milde Straaler fremlokker Livet af Jordens Skjev , bliver , netop nåar den staar hetest , den sorterende , hvis brendende Straaler forvandler Landet til en Orten . Denne Side af Himmeltyset , at det ogsaa stanser , hemmer den Production , det fremkalder , at det er ikke blot Livets , men ogsaa Dedens Magt , fremtreder navnlig hos Syrerne og er personificeret i den fra det gamle Testamente velbetjendte Moloch og den jomfruelige Gudinde Ast ar te , de edeleggende Magter , navnlig den brendende Solhedes og Krigens Guddomme .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014031808082'target=_>Grundtvig, N.F.S., 1876, Kirke-speil eller Udsigt over den christne Menigheds Levnetsløb</a><h4>594</h4>fra den dannede Verden i Romer-Riget til de raa Folk i Norreleden . som den siden har fcerdedes og endnu fcerdes iblandt , og lhvor deM har kummerlig nok fristet Livet , men dog alt gjort kjendelige Trin op ad den lige saa mennestelige som kristelige Livs-Stige . Ogsaa dette Ophold i Folke-Drken , og Opstigningen ad den samme aandelige Livs-Trappe , hvorpaa Nedstigningen fra Jerusalem til Rom flete i den gamle Kristenhed , er jo et stort Guds-Under , der udkrcevede Orkens Omsiabelse til en frugtbar Mark og til en dannet Verden , efter den gamles Forbillede , men med en ny Livs-Kraft , der , menneskelig talt , stilede i samme Retning som Herrens og hans Menigheds <b>Aand</b> , altsaa til Menneffe-Livets Forklaring og Herliggjorelse; men det er et Undervcerk , der ster ved guddommelige Midler paa en menneske-naturlig Maade , hvorved ikke blot den kristne Menighed har faaet Oplysnings - Tid til fcelles med den gamle Menneske-SlEgt , men den gamle Mennest ' e - Slcegt er ogsaa paa Forstandens Enemcerker kommen saa vidt jcevnsides med den ny , at de kan forståa hinanden ligesom den rige Verdensmand og den rige Gudsmand Abraham i Hades og blive sig deres Forhold baade til Gud og Verden og til hinanden klart bevidste . Det er derfor i sin Orden , at der er gaaet omtrent lige saa lang Tid mellem Npostlerne og os , som mellem Ebrceernes Stamfcedre ( Patriarker ) og Kristus , da noget lignende maatte foregaa ved Moses ' Oplcerelse i al Mgypternes <b>Visdom</b> og det jodiste Verdens - Folks aandelige Udvikling til det Trin , hvorpaa en Davids-Datter blev Guds Sons <b>Moder</b> , og hvorpaa lodernes skriftkloge , baade Farisceer og Saducceer , kunde tale med Kristus og med <b>Bevidsthed</b> modtage eller forflyde ham .

<h4>652</h4>Denne Betragtning af Kristen - Livet er derhos saa ojensynlig udtrykt i Legenden om Drythelm , Eremiten fra Melrose , som allerede Beda fortceller som ren Sandhed , stjont han kun har sin <b>Kundskab</b> derom paa tredje eller fjerde Haand , saa den vil altid vare god nok til sit Brug . Sagen er kortelig den , at Drythelm var en gudfrygtig Legmand , som blev syg og dode pludselig en Aftenstund , men vaagnede alt neste Morgen , til stor Forstrekkelse for hans <b>Kone</b> og Born , som sad og gred om hans Seng . Han fortalte da , at hans <b>Sjel</b> virkelig havde veret stilt fra Legemet og i Folge med en Lysets Engel gjort en forunderlig Rejse gjennem de dodes Rige , hvor han grandgivelig betragtede baade Skersilden og Paradis , og fik Lov til at kaste et Blik baade ned i det egentlige Helvede og ind i det salige Himmerig , men blev sendt tilbage til Jorden , for at vandre i et nyt Levned , som han da ogsaa strax begyndte paa , ved at dele hele sin Formue i tre lige Dele , hvoraf hans <b>Kone</b> fik den ene , hans Born den anden , og de

<h4>814</h4>da enten jeg eller en anden kommer og siger , at vi vil scette Middelalderen i sit rette Lys og <b>vise</b> , den var i mange Henseender meget bedre , menneskeligere og dybere end vor Tid og var i alt Fald , <b>aandelig</b> talt , en af vore Forceldre , saa ' vi staar i Grunden i samme Forhold nl Middelalderen som Born til deres <b>Moder</b> , da korser de sig , eller da de fleste har lagt Korset af som noget , der ogsaa borte til den papistiske Surdej i den morke , overtroiske Middelalder , saa trcekker de paa Skuldrene eller stirrer paa os som en gal Kat i Blcest , inderlig overbeviste om , at enten er vi Jesuiter , som vil gjore dem katolske i Hovedet , eller at vi i det mindste , r selv katolske , og de Var da dove Dren og blinde Ojne for alle vore Oplysninger . Alt dette vilde nemlig vceret forebygget eller dog mcerkelig forhindret , dersom den fsrste Dansker , som modte det skæbnesvangre Ord „ Mittelalter " , i Steden for at fmore sig selv om Munden og snakke efter , havde gjort sig den Ulejlighed at sporge sig for eller at tcenke et Ojeblik efter , og derved gjort den lette , men meget frugtbare Opdagelse , ' at „ Mittelalter " paa Tyst er hverken mer eller mindre end „ Mellemalder " paa Dansk ; saa dermed er ikke udsagt det mindste om dens Indhold eller Vilkaar , men kun udtrykt dens Stilling i Tidernes Lob , som Mellemleddet, der langt mindre adskiller den nyere Tid ( Nyaarstid ) fråden gamle ( Oldtiden ) , end den forbinder dem begge med hinanden . Den samme Skjodcslosbed ved Laan fra Tyskerne har bos os magelige Danskere fundet Sted utallige Gange , saa det er Hovedgrunden til den udmcerket forvirrede Tankegang , vt <b>moder</b> snart hos alle dem af vore Landsmcend , der har deres <b>Visdom</b> paa anten eller tredje Haand fra tyste Boger eller deres saalaldte danste Oversættelser; saa den Stjodeslsshed maa vi ved alle Lejligheder revse og beklage , og da iscer ved denne Lejlighed , hvor Talen er om et Tidsrum i Menuesteslcegtens LevnecZlob , lige saa forstjelligt som Follene , der tog Tel deri , og saa langt som fra Folkevandringen , og jeg mener fra Jerusalems

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111501021'target=_>, 1878, streret Verdenshistorie</a><h4>2332</h4>tilbagetrukne Natur tog sig der temmelig maadelig ud . Snart trak han sig mere og mere tilbage fra denne glimrende , men for et folelsesrigt Hjerte lidet tilfredsstillende Kreds , og ikke stort lamgere Varighed fik hans Fortrolighed med Encyklopcedisterne , idet han snart mcerkede , at han ikke var et Barn af den samme <b>Aand</b> som de. Deres kolde og bidende Sarkasmer opwrte ham , og hans dybere Aandsretning kunde ikke npie sig med , at Livets vigtigste Spprgsmaal affcerdigedes med Spot . Hans upolerede , stundom fraswdende Manérer skaffede ham blandt disse forfinede Verdensmcend Vgenavnet  Bjprnen " , og han fjernedes endnu mere fra Berøringer med den store Verden ved sin Forbindelse med Therese Levasseur , en fattig og udannet <b>Pige</b> , hvis ukunstleoe Naturlighed havde indtaget Rousseau i den Grad , at han knyttede en Forening med hende , som skjMt lcenge uden lovlig Bekreftelse dog kom til at vare hele hans Liv igjennem . Han anvendte nu sin Tid dels til det for Livsopholdets Skyld nødvendige Arbeide , dels til enlige Studier , der alene afbwdes ved Spadseretoure og Udflugter i Paris ' Omegn . Adskillige Operaforspg , som dog ikke fik synderlig Fremgang, datere sig fra denne Tid , men det var ogsaa under disse Aars ensomme Ro , at Nousseaus fprste filosofiske Arbeider af stprre Betydning fremtraadte , og det var nu , at han med et blev en evropceisk bewmt Mand . I Aaret 1749 udgav han nemlig i Anledning af et af Akademiet i Dijon fremsat Prisspprgsmaal sin Afhandling vi ^ coul ^ 3 ur leg sciences et leg 2 rt3 . Prisopgaven havde gaaet ud paa , hvorvidt Videnskabernes og Kunsternes Gjenoplivelse har bidraget til at rense Soederne , og det bliver af Rousfeau besvaret bestemt benegtende . Kunst og Videnskab have efter hans Mening sin Nod i Overdaadigheden og Lidenstaberne og ere en usund Kjernes unyttige Frugt , de undergrave Troen og forstyrre Dyden , idet de spre Menneskeheden bort fra dens forste ukunstlede Simpelhed og Uskyldighed og bringe den til at stille Talentet hpiere end Dyden , det godt sagte hpiere end det godt gjorte . Dyden er det hpieste af alt , og den kan ikke vindes ved <b>Dannelse</b> og Forfinelse , men kun ved Tilbagevenden til Naturens oprindelige Tilstande . Men idet han paa samme Tid klart indser, at den Dannelseslpshedens Tilstand , som forekommer ham bedst , for evig er gaaet tabt , indfljcerper han sin Tid , at Videnskaben fra et blot tomt , indholdslsst og stappende Ordkram maa spres ud i det friske , virksomme Liv og der frugtbargjpres til Mennesket ) ed ens Bedste . Uagtet det meget baade uklare og umodne i disse Anskuelser vakte de dog ved sin Originalitet og Dristighed stor Opsigt og lagde den fprste Grund til en ny Anskuelse . Med forpget Klarhed og Styrke fremtrceder denne i den nogle Aar efter udgivne Vl3cour3 3 ur I ' ori ^ ine et le 3 fon6emen3 6 e I ' ine ^ 2 iit6 parmi Is 3 Kc » mme3 , hvori han udtaler den samme Dom over det borgerlige Samfund som tidligere om Dannelsen , nemlig at det er et npdvendigt

<h4>2769</h4>Den ene af disse jakobinske Faktioner , den saakaldte Gir onde , benevntes efter Departementet af samme Navn , hvilket representeredes af de udmcerkede og i Nationalforsamlingen snart almegtige Talere Vergniaud , GensonnS , Guadet og Ducos , men til dem sluttede sig ogsaa Mend fra andre Landsdels . De fleste vare dog fra Sydfrankrige og i det hele udmcerkedes Partiets hele Optreden af det sydfranske Sinds Varme og Fantasirigdom . Den egentlige Ledelse var i Henderne paa den ved sin republikanske Agitation den 17 de Juli bekjendte Brissot , nu Representant for Hovedstaden og mere ivrig end nogensinde for , naar det gjaldt at realisere hans Frihedsdrpmme, at vekke Uro og Forvirring paa alle Kanter . Ved Siden af ham indtog ogsaa en <b>Kvinde</b> , Johanna Ro land , gift med en forhenværende Fabrikinspektpr i Lyon , en fremragende Plads i Partiet . Tenne <b>Dame</b> . der ikke lengere var ung , men besad en usedvanlig Evne til at henrive , gjorde sit Hus til Samlingssted for de fornemste Girondiner og udewede en saa stor Indflydelse paa dem , at man har kaldt hende , Mrondens <b>Sjel</b> " . Hun var lidenskabclig begeistret for Revolutionen , der imidlertid i hendes Tanker endnu var meget langt fra sit <b>Maal</b> , og svermede for en Republik i spartansk og gammelromersk <b>Aand</b> , idet hun oversaa , at selve Forudsetningen for en saadan Statsform , det personlige Forgodtbeftndendes Underordning under Loven , netop var det , som det daverende Frankrige fremfor alt sav ' nede . En saadan Uformuenhed til at bedpmme Sagernes virkelige Stilling var forpvrigt just , hvad der karakteriseres e hele Girondinernes Parti , der i sin übesindige Streben efter et abstrakt Frihedsideal ikke betenkte sig paa at slippe saadanne Krefter los , forn det ikke formaaede at beherske , og ikke nerede nogen Sky for at gjpre Skridt , der tilsidst maatte medfare baade deres egen Undergang og Fedrelandets Fordervelfe . Der maa derfor feldes en streng Dom over dette Partis Medlemmer , naar de stulle bedpmmes som Statsmend , men paa den anden Side var der ogsaa i deres Enthusiasme og Streben efter et idealt <b>Maal</b> noget tiltrekkende , der paa en <b>vis</b> Maade forsoner os med deres Forvildelse og hever dem heit over de pvrige lakobiner . Thi hvad disse strebte til , var ikke andet end i Frihedens

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112201036'target=_>Uhlhorn, Gerhard, 1876, Kristendommens Kamp med Hedenskabet</a><h4>575</h4>Hvad Herren fremstiller som et Tegn og Bevis paa , at han er den sande Messias : „ Evangeliet prcedikes for fattige , " det opfyldes nu i rigt <b>Maal</b> . Det hsrer til en af Tidens Scrregenheder, at de fattiges , betrcengtes , retslsses , undertryttes Kreds er saa stor . Hvilket Indtryk maatte det ej gjsre paa disse , paa alle ejendomslsse , som ikke havde nogen Del i Roms Skatte og Nydelser , paa Smaasolk , som blev traadte under Fsdder , paa Haandvcerkerne , som , fordi de levede af fine Hcrnders Arbejde , blev agtede for intet af den af fin <b>Visdom</b> og Tannelfe stolte antike Verden , om hvilke felv en Plato siger , at deres Liv ikke tjente til andet end til at udsve deres Haandvcerk, og at man , naar de blev syge , maatte overlade dem til deres Skjcebne , da de ikke lcrngere var istand til at opsylde sin Bestemmelse , paa de store Skarer af Slaver i deres for Mennefker uvcerdige Tilstand : hvilket Indtryk maatte det ikke gjsre paa alle disse , naar den fattige lefus , der selv var dsd Slavedsden , forkyndtes for dem , og der i ham aabnedes dem Adgang til et Gudsrige , der omfatter alle , hvor der ikke mere gives Herre og Trcel , hvor ikke mere nogen traedes under Fsdder. Endnu i det andet Aarhundrede fvotter Celsus over , at Uldarbejdere , Skomagere , Garvere , de mest udannede og bondeagtige Folk er de ivrigste Forkyndere af Kristendommen og fsrst bringer den til <b>Kvinder</b> og Bsrn . Men Hedningernes Spot aflcegger imod deres Vilje Vidnesbyrd om , hvilken Magt Ordets Prcrdiken svede , og det , der for den indbildfke Hedning alene er Gjenstand for Spot , det er det jo , hvorfor Herren takker , naar han siger :  leg priser dig , Fader , Himlens og Jordens Herre ! at du har skjult dette for de <b>vise</b> og forstandige og aabenbaret det for de umyndige . la , Fader ! thi det var faa-

<h4>629</h4>Man var nemlig dengang langt fra at foragte Mgteskabet tilbedste for det ugifte Liv . Tvertimod faar LEgteskabet nu den LEre , forn tilkommer det , idet det bliver anerkjendt for en guddommelig Ordning og behandlet i Overensstemmelse dermed. Mgtefkabet sluttes under Menighedens Medviden og Samtykke . Wgteskabsforbindelsen anmeldes for Bistopven og sluttes under hans Velsignelse . LEgteskaber , som stiftedes uden Kirkens Medvirken , anfaaes ej af Kirken for rette VEgteskaber . Der stilles LEgteskabet et hsjere <b>Maal</b> , end det hedenske VEgteskab nogensinde havde kjendt .  Det er " , siger Clemens af Alexandria ,  en Dydsskole for VEgtefolkene til deres egen og deres Bsrns Opdragelfe for Evigheden . Ethvert Hus , enhver Familje maa verre et Afbillede af Kirken ; thi , siger Herren , hvor to er forfamlede i mit Navn , der er jeg midt iblandt dem . " Det er et langt inderligere , dybere Baand , som nu former Mand og <b>Kvinde</b> i det kristelige LEgtestab , den fcelles Tros Baand . Vi sinder hos Tertullian en Lovprisning af det kristelige sEgteskab , hvori han sammenligner det rette kristelige Mgteskab mellem en kristen Mand og en kristen <b>Kvinde</b> med det blandede Wgteskab mellem en kristen <b>Kvinde</b> og en hedensk Mand , og hvoraf vi ikke blot ser , hvor hsjt Mgtefkabet agtedes , men ogsaa kan erkjende , hvad Kristendommen gjorde ud af TEgteskabet , naar den fik gjennemtrcengt det med kristelig <b>Aand</b> .  Hvor skal jeg finde Ord til at skildre Lykken iet Mgteskab , som Kirken stifter , det frembragte Offer bekroefter , og Velsignelsen besegler , som Englene forkynder og Faderen erklcerer for gyldigt ? Hvad er vel ikke det for et Aag for tyende troende , der har st Haab , sn Livsregel , tjener sn Herre . De er Broder

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112301023'target=_>, 1876, Illustreret Verdenshistorie</a><h4>958</h4>hvor Afrika og Europa ncerme sig lynanden mest , havde han opreist sine Stptter ; der , hvor Solen gik til Hvile i Oceanet , spgte han efter Hesperidernes gyldne AMer . Molochs Sindbillede var en Tyr , og han dyrkedes med Menneskeofringen Deraf fremstode Sagnene om den mcegtige og <b>vise</b> Konge Minos paa Kreta , hvis SM , halv Tyr og halv Menneske , maatte mcettes med Menneskekjpd . Lignende Forvandlinger undergik hos Grcekerne de syriske Forestillinger om Maanegudinden Aschera-Astarte , der tcenktes som Ntelkarts <b>Hustru</b> . Snart var hun den  kypriske " eller  kytheriske " Aphrodite , der var stegen op af Havet og i Korinth og paa Sicilien dyrkedes ved HMider , som i vildt Raseri ikke gav de asiatiske noget efter . Snart var hun Europa (  den mprke " ) , en tyrisk Kongedatter , bortrpvet af Zeus i Tyreskikkelse og forgjceves efterfpgt af sin Broder Kadmos (  Merlomdingen " ) , som tilsidst byggede Borgen i Theben og lcerte Hellenerne  den phMikiske Skrift " . Phpnikerne sammenfattede syv af de Meste Guder under Bencevnelsen Kabirer (  de vceldige " ) , og ogsaa disse optoges i den grceske Gudeverden . Paa Samothrake dyrkedes de med hemmelige Mysterier , paa hvis Ndvikling de syriske Prcesters <b>Lcerdom</b> maa have ^ havt stor Indflydelse . Om denne <b>Visdom</b> ved man nu kun lidet , saasom den hele phMikiske Literatur er gaaet tabt . Af nogle grceske Uddrag kan man forståa , at man ogsaa i Kanaans Spstceder gav sig af med Grublerier om Verdens Oprindelse, der i adskillige Henseender mindede om Kaldceernes . Prcefterne indtoge en hpi Plads i Samfundet . I Tyrus kom Melkarts Yppersteprcest Kongen ncermest i Rang . Men Borgerskabet fandt sig ikke iet trykkende Prcestevcelde , ligesaalidt som i et overmcegtigt Kongedømme . Endskjpnt de phMikiske Stceders Forfatning i celdre Tider kun er ufuldstcendig bekjendt , fremgaar det dog af de spredte Oplysninger , at Kongens Magt indskrcenkedes af de mcegtige Kjpbmandsfamilier , som skarpt skilte sig fra den store Masse af Spfolk , Arbeidere , inoflyttede fremmede osv. Stundom , naar Thronen var ledig , valgte Raadet af sin Midte  Dommere " ( Suffeter ) , og i de yngre Kolonier forekom slet ikke Konger , men Regjeringen var helt og holden republikansk . Moderftaden havde for det meste ingen politisk OvertMheo over de nye Anlceg , men Venskabet veoligeholdtes ved gjensidige Forbindelser til Handelens Fremme , og Hjemlandets Templer sik Del i det Krigsbytte , som blev vundet i de fjerne Egne .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100803019'target=_>Geelmuyden, I., 1855, Engelsk Ordbog</a><h4>3542</h4>tuindnc , lom ' - i , ail , « . , Tombak . tumdov , lsm ^ l » t , c / , « . , overgiven , gutagtig <b>Pige</b> , Vildfat . tnmcnt , lo „ l ' - i > al , « . , Halttat . tome , lo , n , « . , Vind , Del ( af en Vog ) , Vog . tunlplun — tnmkin . tuintit , lom ' - lit , « . , Mejse ( Fugl ) . tun , lun , « . , Tun ( <b>Maal</b> A Kommercelest ; Vegt-20Etr . ) . ton , tono , « . , god Tone , fin <b>Dannelse</b> , Mode . tone , lon , « . , Tone , Lyd , Klang , Stemme ; to - , v. , tale ien affetteret Tone ; - d , « ch . , tonende , lydende . ton ? 8 , lo » < ? 2 , « . / il . . Tang . ton ? ue , tuno , « . , Tunge , Tungemaal , Sprog ; Fulf , Stamme : tuKuldllie- , holdeMiind , tie ; to- , v. , snaffe ; stjende ; - d , « ch . , i . Sammens , - tunget ; - leBB , « ch . , maallss ; übenevnt ; - pod , « . , Snatter , Sladder , hank ; to-tie , v. , bindeMunden paa , tvinge til at tie ; - tied , « ch . , som ikke kan tale , tungelam . tonic , ton ' - ill , « ch . , spendende ; spendt , elastisk; som angaar Toner ; - , « . , spendende , nervestyrfendeMiddel ; Gru ! idtone ( iMus . ) . tonnu ? e , 17 / ? i ^ lla , ' , « . , Dregtighed ( af er Skib ) ; Lastrpcnge . tnnsil , lov ' - « il , « . , Tonsil , Halskirtel . lnnBile , lon ' - « il , « ch . , som tan flippes . tonBure , lon ^ - « llsov , « . , Ragen , Tonsur , afraget Sted i Issen ( hos kath . Gejstlige ) . tontine , langlen , « . , Tontine , uorende Livrente, sfor , altfor . too , lss , « elv . , ogsaa , ligeledeS ; ( foran « ch . ) tooi , 1551 , « . , Redstab , Verktoj . to toot , lsst , v. , tude , blefe ; - er , « . , Tuder . tootk , lsslil , « . , s / , i . teetd ) , Tand ; Smag ; - und nuil , af alle Krefter , med Vold ug Magt ; lo tke teetk , i aabenbar Modstand, lige i Ens Ojne ; to cn « t el . lkrow in lde teelk , kaste En i Nesen , bebrejde , forekaste ; in Bpite oltke teetk , naglet al Modstand ; tn Bknw tke teetk , <b>vise</b> Tender, true ; to - , v. , forsyne med Tender , gjere tagget ; gribe ind med Tenderne ( um Hjul ) ; - ed , « ci / . , fursynet med Tender , tagget ; - le3B , « ch . , tandlos ; - v , « ch . , besat med Tender ( om Hjul ) ; - nede , « . , Tandpine ; - drnwer , « . , Tandtretter , Tandlege ; - pick , - picker , « . , Taudstittcr .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015061048109'target=_>Rehfues, Philipp Joseph von, 1834, Scipio Cicala</a><h4>1043</h4>Virkelig kunde det gjelde for en Lykke , at Donna Gaudiosa gik saa vidt i sin Ligegyldighed mod Datteren , at hun neppe spurgte efter hende , naar hun , som sædvanligt skete , vidste, at hun den hele Dag var hos Donna Renata . Den stjsnne <b>Piges</b> herlige Natur havde i hendes Moders Selskab enten maattct vantrives eller blive til Hykleri . Rigtignok vilde Omgangen med hendes Tante alene ikke have vcrret hende nyttigere, og Mclanthos Selskab uden hin vcrre blevet hende virkelig skadelig ; men desto veldædigere virkede det falles Samliv med Begge paa hende » Den barnlige Godmodighed , den fromme Resignation og Kjcrrlighcccn til et stille Liv og kunstnerskc Sysler , som udgjorde Donna Renatas hele Vcesen , begunstigede Udviklingen af den fine Ovindelighcd og vcrkkede Anlcrglne til hver huuslig Dyd tilligemed Sandscn for huuslig Lykke , medens den Gamles vccldige Villiekraft gav hendes Charakteer Styrke , ligesom dennes Tro paa hsiere Forholde imellem Mennesker og Ting mcddeclte hendes <b>Aand</b> et Sving , som beskyttede hende for at hensynke i Hverdagslivet og dets Übetydeligheder. Ak , der bchovcdcs blot en gunstig Stjerne over den unge <b>Piges</b> fjernere Skcrbne , for at gjere ikke blot et sjeldcnt Mynster paa Dyd og <b>Dannelse</b> af hende , men ogsaa en Skat af overstrsmmende nMe for hende selv og Andre .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010011903026'target=_>Becker, Karl Friedrich, 1851, Karl Frederik Becker's verdenshistorie</a><h4>2356</h4>af England betalte , overleveret til hans Embedsmcend . Disse bragte hende til Rouen , hvor Henrik VI . og Regenten dengang opholdt sig . Der blev hun indsat i det store Slotstaarn * ) , blev betynget med Lcrnker og bevogtet af Folk af den simpleste Stand , der saarede hendes Folelse ved Grovheder . Forhaanelser og smudsig Tale , ja endog tillode sig uanstcendlge Mishandlinger . Englcrnderne havde bestuttet hendes Dpd ; Den , der var Aarsagen til alle deres Ulykker i Frankrig , siulde bode for sine Gierninger , de overvundne Feltherrer vilde afvaffe deres Skia ' ndsel i Jomfruens Blod , og Carls Held skulde plettes ved den Oftdagelse , at > det var vundet ved Hclvedeskunster , ved Forbrydelser imod Christendommen og Kirken . I saa Henseende blev nu Processen imod hende begundt af en geistlig Rer , der lededes af Bistopften af Bcciuvais og en Dominicaner , Storinquijuoren af Frankrigs Vicarius i Dioeceset Rouen . Disse to skulde affige Dommen , de ovrige Doctorer i Theologien og Prcrsterne vare kun tilstede som Bisiddcre og for at afgive deres Mening . Man talte henved hundrede , alle Franstmcrnd . Uden at have nogen retskyndig Bistand og overladt aldeles til sin Ungdom og simple Forstand , blev Johanna nu daglig forkort ncesten i sire Timer . Man anvendte de utilladeligste og siicendigste Midler . Paa de underfundige og forvirrende Sporgsmaal , hvormed man uofthorlig bestormede hende , svarede hun til Alles Forundring med en Rolighed , Klarhed, Indsigt og Aandsncerv ^ relse , der langt overgik hendes Ungdom , Kisn og hele <b>Dannelse</b> . Mangt et flaut Sporgsmaal besvarede hun med en , <b>vis</b> Lune . Paa det Sporqsmaal , hvorledes Engelen Michael saae ud , gav hun det Svar :  Som en dygtig , retstaffen Mand , " og paa et andet , om de Personers som havde aabenbaret sig for hende , gik nogne :  Troer I , at Gud ikke siulde kunne skaffe dem Klceder ? " Standhaftig og med Bestemthed , blev hun ved den Erklcering , at hun blot havde . handlet efter de Aabenbarelser , hun havde faaet . Bestaffenheden af disse Aabenbarelser var Hovedgjenstanden for Forhoret , og hendes Dommere bragte tilsidst ud , at hendes Aabenbarelser ikke kom fra Gud , ei heller fra Englene og Helgene , men at Alt var Djcevelsiab og Trolddom. I Scerdeleshed saae man Gudsbespottelse deri , at en <b>Qvinde</b> vilde have bekommet den guddommelige Befaling , at gaae i Mandsklceder. Ogsaa Pariser-Univeisitetct var af samme Mening . Forgieves beraabte Johanna sig paa at demmes af Paven og Conciliet i Basel . Hendes Appellation blev forkastet af Vistoppen , og ved den Trudsel , at

<h4>3131</h4>Men desverre vårede denne fortreffelige Fyrstes Liv kun kort . Han dode allerede 1416 , og hansDod havde i Serdeleshed fordervelige Felger for Castilien . Den unge Konge i dette Rige , forkielet af sin <b>Moder</b> , til hvem Ferdinand havde maattet overlade hans Oftdragelse , var ikke sit store Kald voren . Da han ester hendes Dod ( 1418 ) stod uden Stolte , viste han kun Interesse for de Fornoielser , som lagt og Turnering , Mnsik og Digtekunst — i hvilken han ikke uden Held forsegle sine Krefter — kunde forstaffe ham , men ikke Kraft til at virke i Staten og i Krigen . At faae Indftydelse paa en saadan Konge , maatte fnart blive <b>Formaalet</b> for ergierrige Bestrebelser . En ung Mand af en fornem aragonsk Familie , Alvaro de Luna , der foruden stor Ferdighed i Alt , hvad der interesserede Kongen , tillige besad en kraftig <b>Aand</b> , Snedighed og Mod , havde allerede i Barnelege vundet hans Hjerte . Johan udnevnte ham til Rigsconnetable og var snart aldeles i hans Hander . Men den afdode Konge af Aragoniens yngre Sonner , Johan , Henrik og Sancho , der ved at arve Familiegodser vare blevne megtige Vasaller af Castilien , misundte ham denne Stilling . I Forstningen benyttede den listige Alvaro den Skinsyge , Vrodrene nerede

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009082403003'target=_>Krummacher, F.W.., 1857, Elisa</a><h4>1055</h4>Elisas Bud ankommer til Samaria , iler til Paladset , fremstiller sig for Kongen og udforer sit Mrende . — loram horer hans Vudstab med Fornoielse ; han seer sig derved besnet fra en hemmelig Forlegenhed , og dette lader ham let forvinde det Bittre i Elisas Ord . Han lnder den syge Feltherre sige , nt i lericho boer en Prophet ; til hnm mnntte han umage sig hen . Aldrig gjorde en Veiviser af Tree en mere foronstet Tjeneste , end denne . Naamans Glede over denne uventede Efterretning er übeskrivelig ; Vudstabet lyder for ham , som den sig nermende Nedningsmands Aarestag lyder for en Skibbruden imellem Bolgernes Brusen ; som den frelsende Stemme, der raaber paa hans Navn , lyder for En , der er indesperret i en dyv Skakt . " Om en Prophet " , tenker han , "talete jo ogsaa den lille <b>Pige</b> i Damascus ! " og han bliver nesten til Mode , som hine <b>Vise</b> fra Osterland maae vere blevney da de ved deres Udgang af Jerusalems Porte atter saae sig lyste af hiin Stjerne , der stulde lede dem til <b>Maalet</b> for al deres Lengsel , alt deres Haab . Hans sode Haabs allerede nedtraadte , og sorgeligt henvisnede Sknbning er ved dette gode Budstabs milde Foraarsregn , i et Oieblik atter opblomstret i den stjonneste Glands , og hans Tillid til en ner Redning er bleven vissere og gladere , end den nogensinoe for hnr veret . M , paa enhver Vandring , den gaae hvorhen den vil , kommer det kun derpaa an , at Gud bod os drage , og at Han satte os i Bevegelse ; da kunne vi itte forfeile <b>Maalet</b> . Vi Alle , saa Mange af os , som hore Ehristo til , ere paa Vandring , ligesom Naaman , og ikke alene til hiin Stad , hvortil der var en Grund ; men til endnu mange andre <b>Maal</b> as mange Slags . Vi Predikanter

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014082208103'target=_>Bunyan, John, 1859, Pilegrims vandring</a><h4>528</h4>Forfcedre Byen forst sit sit Navn ; ligesaa Smigrer , Dobbeltvei , Alting , og Proesten i vort Sogn , Tvetunget , var min Moders egen Farbroder , og reent ud sagt , saa er jeg bleven en temmelig anseet Mand , endstjont min Oldefader kun var Baadsmand , som saae mod en Vei , og roede ad en anden ; ogsaa jeg har faaet det meste af min Giendom ved samme Forretning , CH rist . Er du gift ? Egennytte . Ja , og min <b>Kone</b> er en meget dydig <b>Kone</b> , hvis <b>Moder</b> var ligesaa dydig . Hun er en Datter af Fru Forstillelse . Fordi hun er kommen af en meget hcederlig Familie , har hun ogsaa saamegen <b>Dannelse</b> , at hun veed , hvorledes hun stal opfore sig imod Alle , saavel mod Fyrsten som mod Bonden . Det er rigtignok sandt , vi afvige noget i Religionen fra dem af de strangere Slags , men dog kun i to übetydelige Punkter : Forst , „ vi strcebe aldrig mod Veir og Vind ; " for det Andet ere vi meest ivrige , naar nemlig Religionen gaaer i Solv-Tsfler ; vi synes meget godt om at " spadsere med ham paa Gaden , naar Solen skinner og Folket giver ham Bifaldsraab . Derpaa gik Christen lidt tilside til sin Ven Haabefuld og sagde : „ Det falder mig ind , at denne er en <b>vis</b> Egennytte fra Byen Godtsnak , og dersom det er ham , saa have vi sandelig en saa stor Skjelm i vort Selskab , som der kan findes her tillands . " Da sagde Ha abefuld : „ Sporg ham , jeg troer ikke han stammer sig ved sit Navn . " Christen gik da til ham igjen , og sagde : „ Du taler , som om du skulde vide mere , end hele Verden , og om jeg ikke tager Feil , saa troer jeg , jeg saa halvt veed , hvem du er ; er ikke dit Navn Egennytte fra Godtsnak ? "

<h4>1301</h4>sig sire saadanne Drenge , som disse . Jeg haaber , at de ville gjore deres Faders Navn Wre og tmde i hans Fodspor , og naae deres Faders <b>Maal</b> . Behj . De ere virkelig brave Born , og de synes at vcrlge deres Faders Veie af ganske Hjerte . Gajus . Det var ogsaa min Mening , hvorfor Chnstmes Familie stedse synes at udbredes og blive talng paa Jorden ; lad derfor Christine udsoge nogle Mger for sine Sonner , med hvem de knnde blive trolovede, at deres Faders Navn og Stammødres Hnus aldrig maa glemmes i Verden . Mrligh . Det vilde vcere beklageligt , om denne Familie skulde falde og uddo . Gajus . Falde wn den ikke , men formindstes kan den ; derfor maa Christine folge mit Raad , oq da vil den vedligeholdes . Det fornoier mig at see dlg Christine og din Veninde Barmhjertig her sammen , et elskværdigt Par ; og tor jeg raade , da tag Barmhjertig iet noiere Beflcegtstab til dig . Dersom hun VU , saa lad hende faae din ældste Son Matthams hvorved Cftersicegten vil vedligeholdes paa Jorden ' Saaledes blev dette Parti stiftet , og de bleve siden gifte , hvorom Mere i det Folgende . Gajus sagde videre : . Jeg vil nu tale tilQvindernes Forsvar . Thi ligesom Dod og Forbandelse kom md r Verden ved en <b>Qvinde</b> ( 1 Mos . 3. ) , saaledes kom ogsaa Liv og Salighed ved hende . „ Gud udsendte sin Son , fodt af en <b>Qvinde</b> , " ( Gal 4 4 ) Ja fvr at <b>vise</b> , hvormeget Gftersla > gten asstyede Moderens Handling , oustede <b>Qvindekjonnet</b> i det gamle Testamente Bom , da muligen denne eller hiin <b>Qvinde</b> kunde blive <b>Moder</b> til Verdens Frelser . Hertil maa jeg desuden lcegge , at da Frelseren var kommen til Verden glcedede <b>Qvinderne</b> sig , fmend nogen Mand eller En- Vui , l , uuj Pillegrimsv.nitring , 1 ! l

<h4>1437</h4>men var for undseelig til at bede derom . Hendes Svigermoder spurgte da hende , hvad der fattedes hende ; thi hun saae ud , som hun ikke var ret vel . „ Der hamger et Speil , " sagde Barmhjertig ; „ inde i Spisevarelset, som jeg ikke kan faae ud af mine Tanker , og dersom jeg ikke faaer det , troer jeg det ikke gaaer mig vel . " „ Jeg vil omtale det til Hyrderne , " sagde Moderen , " og de ville vist ikke nagte dig det . " „ Men jeg stammer mig ved , " svarede hun , „ at disse Folk stulle vide , at jeg har nogen Loengsel . " „ Nei , min Datter , " sagde Moderen , „ det er ingen Skam , det er en Dyd , at langes efter saadant . " „ <b>Moder</b> , " sagde Barmhjertig , „ vil du da sporge Hyrderne , om de ville scelge det . " Dette Speil var eet af det kunde paa den ene Side fremstille et Menneske med alle dets Ansigtstrak paa det noiagtigste ; og naar man vendte det om paa den anden Side , saa viste det ligesaa tydeligt Ansigtet og Billedet af PillegrimmeneZ Fyrste selv ; ja , jeg har hort Folk sige , at de have seet Torne- Kronen paa hans Hoved ved at see i dette Speil ; de have deri ogsaa seet Hullerne i hans Hcender , i hans Fodder og i hans Side . Ja dette Speil har den Fuldkommenhed , at det kan <b>vise</b> ham , hvordan man end har Lyst at see ham , cnteu levende eller dod , enten paa Jorden eller i Himlen , enten i hans ydmygelses eller i hans Ophsielses-Stand , enten kommende for at lide eller for at regjere . Christine gik derfor tilside til Hyrderne , hvis Navne vare <b>Kundskab</b> , Erfaring , Aarvaagen og Oprigtig , " ) og sagde til dem : „ Der er een af mine Dottre , som er soanger , og som har Lyst til Noget , som hun har seet i Huset , ' og hun troer , at

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013012824012'target=_>Holberg, Ludvig, 1873, Ludvig Holbergs Epistler</a><h4>1329</h4>som de hcesligste blandt den heele Hunde-Nation . Hvad Smagen udi Skrifter og <b>Lcerdom</b> angaaer , da grunder den sig og paa antagne <b>Moder</b> : Nu holdes best grundet , nu et andet ; nu ad- mii-61-68 en nu en Nen ton , og nu opkaages igien den fsrstes svltema : Nu er en ? lato paa Thronen , nu en og viser hvilken foranderlig Skiebne den sidstes <b>Lcerdom</b> haver havt , i det den nu haver vceret anseet som daarlig og ugudelig , nu igien som ypperlig og fast guddommelig . Efterdi det nu er saaledes beflaffet med Menneskers Smag , saa gisre de Hsyviise Herrer bedre ikke at ivre sig saa hceftigen mod ovenmeldte Qvinde-Dragt , helst saasom vi , der ere af det andet Kisn , derved intet lide . Mit Raad stulde heller vcere at arbejde paa de Perukkers Afskaffelse , som <b>Fruentimmer</b> nyeligen have antaget ; thi derved diares Mands-Personer , som alleene hidindtil have voeret i af saadanne Hoved- Smykker , og som kand <b>vise</b> nogle 100 de Aars Hcevd derpaa . Det er Magtpaaliggende , at saadant betimeligen forekommes , saasom det er at befrygte , at hvis <b>Fruentimmer</b> herudi faaer fri Hcender , de vil ssge at bersve os andre af vore Zirather , og som i scer vise

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009082503009'target=_>, 1858, Bibelen eller Den Hellige Skrift</a><h4>1468</h4>Hustru , og at det skulde antydes ham , at Kongen ikke nu trcengte til hans Tjeneste . — Men Una laa for Doren til Kongens Huus med alle sin Herres Tjenere d . e . den kongelige Livvagt . Det var muligt , at Uria havde erfaret noget om Davids Mgtcstabsbrud , som mange maatte kjende til ; dog berettes der Intet herom , men det synes som den hellige Forfatter netop derved vil lade David fremtrcede i et vcerre Lys , at han havde med en saa from , oprigtig og sin Herres Tjeneste saa ganske hengiven Mand at bestille . — Arken : I en saa vigtig og farlig Krig havde de medbragt Pagtesarken ligesom tilforn mod Philisterne. — Israel og luda ligge i Teltene d . e . Lovhytter . — Bliv og idag her , saa vil jeg unorgen lade dig fare : Stedse videre og dybere griber Synden om sig : Medens David ikke kan naa sit <b>Maal</b> ved at skjule den Gjerning , for hvilken Loven havde sat Dodsstraf, vil han nu rydde Uria afvcien . — Han gjorde ham drukken , for at han paa denne Maade stnlde gjore , hvad Listen for ikke havde formanet ham til . — Scrttcr Uria foran , der hvor Krigen er stcerkest , og vender tilbage fra ham , at han kan blive slagen og do . Han var saa <b>vis</b> paa , at denne Befaling blev udfsrt , at han ikke engang behsvede at ncevne , hvori Urias Forbrydelse bestod . — Ta sendte loab hen og lod give David tilkjende alle Krigens Sager . loab viser sig ved denne Anledning ligesom overalt raa og voldsom og som den , der ikke undslaar sig for nogensomhelst Gjerning ; den übetingede Lydighet » , han skyldte sin Konge , kan neppe have vcrret hans Bcvceggrund; han maatte ane Sammenhangen eller have erfaret den gjennem Nygtet , og i en for hans lave Bctragtning ligegyldig Sag vilde han gjere sig ' David til Ven . — Hvo flog Abimclech , Icrnbbeseths Sen ? I det hebraiske Sprog betyder bos ^ Gtd Skjcendsel ; dette var en senere Forandring as Navnet lerub Baal , der udtrykte Folkets Afsky for Afguden . Fortcellingen berettes Dom . 9 , 13. — — V . 22 — 24. Sendebudet tog sig Sagen meget let ; David indsaa Sammenhangen meget snarere , end loab havde ment . — Ter Sorgen var gaaen over , sendte David hen og lod hente hende til sit Huus . Hvor lcrnge en Enkes Eorgetid vårede , om det var over de scedvanlige syv Dage ( 1 Mos . 50 , 10. 1 Sam . 31 , 13 ) berettes os Intet . lalfald gjorde han Sagen af saa kort som mulig , da der i den udvortes Sorg aabenbar ikke laa nogen Anger over den begangne Forbrydelse.

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014062048057'target=_>Vetlesen, Jacob Henning, 1849, Bergens Sommer</a><h4>2042</h4>der paa Grund af den bekjendte Stemme spamdte minNysgjerrighed paa det Hoieste . Skal jeg lade ham drage bort , blev der spurgt , hvorpaa Svaret lod : Ja lad ham kun det . Men vil jeg ogsaa faa ham at se igjen , spurgte den bekjendte Stemme . Engang vil I faa ham at se . Men var det ikke bedre at jeg ikke lod ham reise , spurgtes der igjen . Nei , derved vil han komme til sin Net , var Svaret . Ser I mere , spurgte den Naadsogende . Nei Intet mere . I tror altsaa han vil komme til sin Net . Ja svarede den gamle og hostede saa det peb ud igjennem Ljaaren , der ei var mange Fod fra mit Hoved . Jeg Horte at der blev taget Farvel under mange gjensidige , halv hoit udtalte Forsikkringer og Velsignelser . Jeg skyndte mig ben til Skoven og neppe var jeg vel skjult der , for jeg saa Halvards <b>Hustru</b> , Nagnhild den Trondske , som hun kaldtes , fordi hun var af en mcrgtig Slcrgt der Nord fra , komme ud af Hytten . Jeg havde altsaa ikke taget Feil , det var Ragnhilds , den stolte Ragnhilds Stemme . Hun gik med et Fjed saa hurtigt og let , som om hun i Hytten havde faaet Helsebod for mange Aars Lidelser . Hun forsvandt i Skoven paa den modsatte Side af den hvorpaa jeg stod . En übeskrivelig Lyst til at trcrnge ind i Fremtidens dunkle Egn , ved Hjcrlft af den <b>vise</b> <b>Kvindes</b> <b>Kundskaber</b> overkom mig . Jeg ncrrmede mig Hytten og traf den Gamle siddende boirt over en Kjedrl , der kogte paa Ildstedet . Hun reiste sig , da hun saa mig , og jeg blev forstrækket over hendes Hoide . „ Hvem trceder ind her , uden at have erholdt Tilladelse , " spurgte hun vredt . Jeg bad om Forladelse for min Paatrnngenhed og undskyldte mig med min Begærlighed efter ogsaa at erfare hvad Fremtiden kunde bringe mig . Hun betragtede mig igjennem sine rode Oine , idet hun torrede Nogen af dem . Du ser ud til en loierlig En , sagde hun , og tog Kjrdelen af Ilden . Det gaar ei saa hastigt at erfare sin Skjebne , som Du synes at forudscrtte , jeg maa vide mange Ting for jeg kan sige Dig Noget. Hun talte da meget med mig og jeg synes endnu jeg kjender

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111500002'target=_>Poulsen, Vilh., 1880, Fra Reformationstiden</a><h4>1836</h4>fsler mig « stikket til alt ; alligevel vil jeg , da jeg ved , hvormeget du snfler det . samle Styrke til i Korthed at fortcelle dig om hendes Endeligt , Lige fsr hun dsde , og som hun vaagnede op af en Slummer , havde hendes Aasyn et Udtryk af forklaret Blidhed og stille Glcrde . Jeg traadte til og spurgte , hvorfor hun smilede saa sodt .  leg saa " , svarede hun .  just nu , da det var stille , et Sted , fyldt med et deiligt , klart Lys " ; hun formaaede af Mathed ikke at sige mere .  Velan " , sagde jeg ,  v < rr frimodig , min <b>Hustru</b> , du stal bo i hint stjsnne Lys " . Hun smilede atter og nikkede svagt med Hovedet , og lidt efter sagde hun :  leg er saa glad " . Hun talte ikke mere , for hendes Blik omstsredes; da sagde hun :  leg kan neppe kjende dig , men alt omkring mig synes fuldt af deilige Blomster " . Dette var hendes sidste Ord . Lidt efter , som om hun hensank i en sod Ssvn , opgav hun sin <b>Aand</b> . - Tidligere havde hun ofte erklceret med stcrrk Forvisning , at hun onskede intet hellere end at ds og verre med Kristus , hvis store Naade mod hende hun aldrig , naar hendes heftige Sygdom tillod det , ophorte at erkjende , fordi han havde oplyft hende med sit Ords <b>Kundskab</b> og havde holdt hendes Sjcrl borde fra Verdens Fornoielser og indgivet hende en inderlig' Lcengsel efter det evige Liv ; heller ikke betcenkte hun sig i alle Samtaler med hende at kalde sig et Guds Barn . Intet tiltalte hende mindre end at forssge at troste hende ved at udtale Haab om , at hun vilde komme sig af sin Sygdom . Gud , sagde hun , havde bestemt hendes Livs Lsv ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008092203008'target=_>, 1858, Livsbilleder af Guds Riges Historie</a><h4>357</h4>sine ualmindelige Gaver , var Ydmyg , besteden og venlig mod Enhver. Og for ret at dygtiggjyre hende som et Guds Menneske til al god Gjerning fyrte Herren hende ogsaa i Korsets Skole ; hendes Foreldre behandlede hende med en paafaldende og fast uforstaaelig Strenghed . Hendes Fader , Hertugen af Suffolt , var en Mand , der ved sin <b>Dannelse</b> , sin <b>Lerdom</b> og sin Fromhed i roligere Tider vilde have erhvervet sig almindelig Agtelse ; den opryrte Tid , i hvilken han levede , bragte ham ind i Forviklinger, hvilke han ikke var voxen . Hvorfor Foreldrene behandlede deres ndmertede Datter med en Strenghed , der tidligen herdede hendes blyde Sind , er übekjendt . Naar Johanne saaledes var nedbyiet og bedryvet over übillig Dadel , da vendte hun med dobbelt Lengsel tilbage til sine Studier ; hendes elskede Lerere Aylmer og Harding , Mend af anerkjendt <b>Lerdom</b> , vare hendes Tilstugt ; under dem gjorde hun de store Fremstridt . Sjeldent kom hun til Hove , men boede for det Meste paa Landet paa sin Faders Gods Broadgate i Leicestershire . Der fandt den berymte Professor i Cambridge Roger Asham hende engang ( 1550 ) lesende Platos Phedon i Originalsproget , medens den hele yvrige Familie var paa lagt , og det med en saa yiensynlig Glede , at den navnkundige Lerde ikke kunde glemme Indtrykket af den unge <b>Pige</b> , som underholdt sig med Oldtidens viseste Mand om <b>Sjelens</b> Udydelighed . Dog forundtes det hende ikte lenge at leve i landlig Ro og Forborgenhed, da hun blev gift med Northumberlands fjerde Syn , Guildford Dudley ( i Mai 1553 ) ; Edvard gav hende et rigeligt Udstyr ; det var den sidste Lysstraale i hans Liv . — Kongens Dyd blev holdt hemmelig i nogle Dage , thi nu begyndte det underfundige:: forberedte Drama , i hvilket den uskyldige Johanne var ndseet til at spille Hovedrollen , og hvori hun maatte falde som et Offer . Johanne var med sin unge Gemal i Durhamhouse , da hendes Fader og Svigerfader kom til hende og aabcnbarede hende med megen Hyitidelighed Kongens Villie angaaende Udeluttelsen af Marie og Elisabeth og hendes Udnevnclse til

<h4>385</h4>kjydelige Synder , den sorteste Feighed og Trolyshed , Gift og Snigmord herskede allevegne " . Men at Evangeliet er megtigt til at bevare en <b>Sjel</b> reen , om den endog er omgiven af de styrste Besmittelser og lever i den meest fordervede Tid , dette viser klart og stjynt den edle Dronning af Navarra , Johanne a f Aldret , som hyrer til Rekken af de hyibaarne Fyrstinder fra Reformationstiden , i hvilke den sande Tro ndfyrte sit helliggjyrende Verk . Det er den lyseste Side i hiin Tids mange Skygger , at den nye Lere fandt Indgang hos mange af Adelen , hos Magistraterne , som overhovedet hos dem , der vare de meest fremragende i <b>Dannelse</b> , Anseelse og Videnstab ; disse yvede en stor Indflydelse paa deres Tidsalder . Blandt disse indtager den nevnte Dronning en hyi Plads . Hun fik en stor Opgave i Livet og lyste den med et Mod , om hvilket Calvin siger , at det bestemmede Mend , og hun indtog sin Plads paa Verdens Stueplads , som den frommeste <b>Kvinde</b> .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100803011'target=_>Becker, Karl Frederik, 1851, Karl Frederik Becker's Verdenshistorie</a><h4>903</h4>Et vildt , i Huler og Kloster boende Folk , som Nogle mene , vare Pelasgerne vistnok ikke , men derimod et agerdyrkende og i Besiddelse af en <b>vis</b> acmdig <b>Dannelse</b> . Dem tilskriver man de saa kaldte cykloftiske Mure , af hvilke der endnu den Dag i Dag findes Levninger saavel i Grekenland som i Italien . Det er Mure , som ere sammensatte af uhyre Eteenblokke , der ere passede saa noie i hverandre , at de , uden at vere forbundne med Kalk , danne et urokkeligt Heclt , og da de Gamle ikke troede , at sedvanlige Mennesier kunde have fuldendt saadanne Vcerker , erklcrrcde de Opbyggerne for Kemper ' eller Cykloper . Pelasgernes fortrinligste Gudsdyrkelse var Zeus ' i Do dona , i det nordlige Epirus , hvor der var et beromt Orakel . Der var mange Maader at adsporge det paa . Snart gik de dertil ansatte Prcrstinder til den hellige Lund under en gammel Eeg , som blev anseet for guddommelig , og lyttede til Bladenes Susen og en tott forbi rislende Kildes Surren ; snart tydedede Gudernes Villie af den Klang , som opstod , naar flere Kobberbekkener , der hang rundt omkring Templet udsatte for Vinden , stodte mod hverandre . Om Oprindelsen til dette Orakel fortalte man i Dodona , at der en Gang fra Theben i Wgyvten var sioitt to sorte Duer , som havde nedladt sig , den ene i Libyen , den anden i Dodona ; den sidste havde der sat fig paa en Eeg og med menneskelig Stemme befalet dem at oprette et Orakel der . Men Herodot , som forteller dette , tilfoier allerede selv , ledet dertil ved en Etterretning , han havde erholdt af Piestcrne i Theden , Udtydniugcn af denne Mylhe , at Dnerne nemlig havde veret to hellige <b>Qvinder</b> fra MgVfttcn , der havde stiftet disse to Orakler . Maastee et eldgammelt Forsog af de egyptiske Pester paa at forplante deres Cullus derheu .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011050604121'target=_>Lauridsen, Peder, 1865, Catechismus</a><h4>453</h4>Cap . : Alligevel bliver det uretfcerdige Gods i den Ugudeliges Hus , og den hadelige ringe Epha . Eller skulde jeg samtykke i den urette Vcegtstaal eller den falske Vcegt i Scekkene ? med hvilket de Mr meget Uret . Salomon siger ogsaa : Mangeslags Vcegt og <b>Maal</b> ere vederstyggelige for Herren , og en falsk Vcegtflaal er ikke god . ( Prced . 20 ) . Det fjerde Slags Tyve ere de , som hjcelpe andre , enten lcere de dem , hvorledes de stulle stjcele , eller <b>vise</b> dem paa Vei , eller giver dem Kundstab til Tyveri , eller befale nogen at stjcele som Forceldre eller Husbonder tilholde deres Bern , Tyende eller andre til Tyvehandel . Derom siger Salomon: Hvo , som haver Del med Tyve , hyrer Bander, og giver det ikke tilkjende , han hader sit Liv . ( Prced . 29 ) . Det femte Slags ere de , som samtykke i Tyveri, enten fyrend det bliver begaaet eller i Gjerningen, etter efterat de har stjaalet . Imod hvilket Tvveri vi maa lcese en fljsn <b>Lcerdom</b> , ( Tob . 2. ) : Der Anna , Tobias ' s <b>Hustru</b> hentede et Kid hjem , og Tobias , hendes Husbonde hsrte det brcege , sagde han , se til at det ikke er stjaalet ; giver den rette Herre det igjen , thi os ber ikke at cede af stjaalet Gods eller rore det . Saa byr os ikke at hcele eller dplge med Tyven ; thi man siger : Hceleri og Dslg er vcerre end Tyveri . ( 1 Moseb . 36 ) . Saa dulgte Rachel Afguderne og lagde dem under Kamelernes Strselse , og fatte sig der ovenpaa , og Laban randsagede det ganske Paulun , og fandt intet. Da sagde hun til sin Fader : Min Herre vcer el vred , jeg kan ikke staa op mod dig , fordi det

<h4>751</h4>Mnd , at han er en Herre , oss troer i dit Hjerte lt Gud opvakte ham fra de Dode , da bliver du alig . ( 1 Cor . 1 ) . Dog denne Artikel strider ganske mod menneskelig Forstand og Fornuft , som Paulus aler : Iyderne ceste Tegn og Grcekerne sporqe eser <b>Visdom</b> ; men vi prcedike den korsfcestede ) esum , som er Iyderne en Forargelse , og Orceerne en Daarlighed ; men dem som ere kaldede , > aade ' leder og prcedike vi Christum Vuds Kraft og Guds <b>Visdom</b> ; thi Guds Daarisshed er visere end Menneskene ere , og Guds Skrsirlighed er stcerkere end Menneskene ere . I se , jcere Bredre , Eders Kald , der er ikke mange <b>vise</b> ' ster Kjpdet , ikke mange Mcegtige , ikke mange Veliaarne ere kaldte og udvalgte ; men hvad forn daarigt er for Verden , det udvalgte Gud , at han ) ilde bestjcemme de <b>Vise</b> . Og Gud udvalgte det om strpbeligt er for Verden , at han vilde bekjcemme det Stcerke , og Gud udvalgte ucedle ! ing for Verden , og det Foragtede , og det som Zntet er , at han vilde tilintetgjsre det , som Noget : r , paa det at intet Kjod stal rose sig for ham . Nl denne Artikel at stadfceste bor det os at agte : Hprst , Vidnesbyrd for Opstandelsen , baade i ud.rykte Ord og Figurer . Derncest , Tegn og aaben>ar mundt iq Bekjendelse , strax i Christi Opstan>else. For det Tredie , Vidnesbhrd efterat han er > pfaren til Himmelen , at det visselig saa er steet , ned sin <b>Lcerdom</b> , Gavn og Trost . Det fyrste , saa aler Gud ( 1 Moseb . 3. ) : <b>Kvindens</b> Sced stal spn)erknuse Slangens Hoved , og Gud sagde til Abra,)am ( 1 Moseb . 12 ) : I dig stulle alle Sloegter paa

<h4>849</h4>vi den vcerdige Hellig <b>Aand</b> vor Helliggjorelse , at naar vi med gudfrygtige Z3ren hyre Guds Ord , saa opvcekker han Trost og Husvalelse i vore Hjerter, imod Synd , Sorg , Modgang og alle vore Fiender , Ded og Ulykke . ( Apost . Gj . 5 ) . At den Hellig <b>Aand</b> er en sand Gud , giver St . Peder noksom tilkjende , der han sagde tilAnanias : Du haver ikke lyvet for Menneskene , men for Gud , og sagde til Saphira Ananice <b>Hustru</b> ; hvi ere I da blevne samdrcegtige at fritte Herrens <b>Aand</b> . Og Moses taler i Skabelsens Bog , og siger : Guds <b>Aand</b> svcevede over Vandene . ( 1 Moseb . 1 ) . Og David bekjender Guds Almcegtigheds <b>Aand</b> , idet han siger ( Psal . 139 ) : Herre , hvor skal jeg gaa hen fra din <b>Aand</b> , hvor stal jeg fly bort fra dit Ansigt? Farer jeg til Himmelen , da er du der , gjsr jeg min Seng i Helvede , se , da er du der ; tager jeg Morgenstjernens Vinger og bliver ved det yderste Hav , saa skulde dog din Haand fsre mig der , og din hpire Haand holde mig . Og i <b>Visdommens</b> Bog 1 Cap . : Verdens Kreds er fuld af Herrens <b>Aand</b> ; det er , at den Hellig <b>Aand</b> med Gud Fader og Son styrer , regjerer , bevarer og opholder alle Ting , baade i Himmelen og paa Isrden. Som ogsaa lob bekjender ( lob . 33. ) : Guds <b>Aand</b> haver gjort mig , og den Almcegtiges <b>Aand</b> haver givet mig Livet . Heraf kunne vi vel forståa, at den Hellig <b>Aand</b> er en sand Gud . Det andet om den Hellig <b>Aand</b> , at han er den tredie Person i Guddommen , have vi ingen bedre Forklaring end den , som er steet i Jesu Christi Daab , ( Matth . 3. ) , der Gud Fader talede af

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009022604082'target=_>Utheim, J., 1880, Sagn og Fortællinger af Oldtidens Historie</a><h4>1117</h4>Vi har saaledes skildret de af Solon grundlagte Statsformer i deres senere fyldigere Udvikling . Men Solons Lovgivning omfattede desuden en Mængde Love og Bestemmelser, som angik Folkets Liv og Seder , f . Eks . Bestemmelser om Agerbruget , om Haandverksdriften og Handelen , om Fadermyndigheden , Arveretten , Opdragelsen , om <b>Kvindernes</b> Luksus o . s . v. I alle disse Bestemmelser raader den samme retfærdige , frisindede <b>Aand</b> , som er et Særkjende for Solons Lovgivning . I Lykurgos ' Stat forsvandt Mennesket for Borgeren , der blev hemmet i sin frie Udvikling af de Baand , hvori Staten indsnørede ham . I Solons Stat derimod var der ladet den enkelte vidt nok Spillerum til at udfolde sine Evner . Medens Lykurgs Lovgivning lagde Beslag paa og udviklede Folkets Kraft fortrinsvis kun i én Retning , nemlig at danne Borgerne til fremragende Krigere , var Solons <b>Maal</b> at udnytte alle Kræfter blandt Folket , at mane Borgerne til Kappestrid ogsaa i Fredens Sysler og fremme en alsidig Udvikling og Dannelse. Medens Sparta forbød sine Borgere at arbeide , gjorde Athen dette til en Pligt og straffede dem , som gik ledige . En Søn var ikke pligtig til at underholde sin Fader i hans Alderdom , nåar denne ikke havde ladet ham lære en Haandtering .

<h4>1528</h4>Da hans <b>Moder</b> Aga ri ste bar ham under sit Hjerte , drømte hun , at hun skulde føde en Løve , et Varsel om kommende Storhed . Gjennem sin Fader Xanthippos , Seierherren ved Mykale ( Side 305 ) , tilhørte Perikles en af Athens ældste Familier , gjennem sin <b>Moder</b> var han en Alkmæonide. Da hans Fødselsaar falder omkring 494 , har han som Gut oplevet sin Fædrestads Trængselstid , og medens han voksede op til Yngling , saa han Athen stille sig i Spidsen for et mægtigt Forbund af Øer og Kyststeder , og han erkjendte , at det var hans Fædrestads Opgave at <b>vise</b> sig en saadan Stilling værdig . Og til at arbeide med til dette <b>Maal</b> var Perikles kaldet ikke alene ved sin Fødsel, men ogsaa ved de lykkeligste Anlæg ; thi han var af Naturen rigt udrustet . Det ædle Ansigt med den lige græske Næse , de tankefulde Øine og den fint dannede Mund vidnede om Alvor og Kraft , og paa hans Pande havde Klogskaben sat sit übedragelige Merke ; livlig og rig paa Ideer som Themistokles var han i hele sit Væsen fra Ungdommen af langt mere samlet og harmonisk end denne . Og denne rige <b>Begavelse</b> blev fra den tidligste Ungdom pleiet og ledet af de bedste Lærere ; gjennem Omgang med Filosofer , med Digtere og Kunstnere søgte Perikles Næring for sin brændende Kundskabstørst , og denne forædlende Omgang skaffede ham ikke alene Indsigter, men gav ham ogsaa den Følelse af <b>aandelig</b> Frihed,

<h4>1537</h4>Perikles ' Privatliv var tilbagetrukket ; det var tarveligt og maadeholdent , beregnet paa og værdigt til at vinde alles Agtelse . Mere end de fleste Athenere formaaede ogsaa Hjemmet at fengsle ham ; thi det smykkedes af en <b>aandfuld</b> <b>Hustru</b> , den skjønne Milesierinde As pas ia . Denne <b>Dame</b> stod høit over de attiske <b>Kvinder</b> i <b>Dannelse</b> , og hun forstod at samle i sit Hus en udvalgt Kreds af begavede Aander . Dette bidrog vel ogsaa sit til , at Perikles kun tog liden Del i det atjiéniske Selskabsliv ; ja det fortelles endog , at han kun en eneste Gang udenfor sit Hus skal have bivaanet et festligt Gilde , som han endog forlod , da Drikkelagets støiende Munterhed begyndte .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111400038'target=_>, 1878, streret Verdenshistorie</a><h4>1475</h4>- Henrik IV , en af de stprste af alle Frankriges Herskere , var fM den 4 de December 1553 i den lille Stad Pau i det af Pyrenceernes Bjerge opsyldte Landstad Bearn . Han var en Spn af den svage og vankelmodige Kong Anton af Navarra og dennes sjelssterke , heltemodige <b>Hustru</b> , Dronning Johanna d ' Albret . I sine yngre Aar stod han under en omhyggelig Ledelse af sin <b>Moder</b> , en af sin Tids fortreffeligste <b>Kvinder</b> , og de bedste af de Egenstaber , der prydede ham , vare en Arv fra hende . Han sik uden Tvivl en god <b>Dannelse</b> og oplcertes , som det s ^ mmede sig for ham , i enhver ridderlig Idrcet . Hans Ungdom henflpd under de franske Borgerkriges Vaavenlarm , og han saa dem paa tilstrekkelig ncert Hold til at kunne sige , at han var opfpdt i en Leir . Under det umiddelbare Indtryk af disse Kampe modnedes han til Mand . Begivenhederne bragte ham snart til hans egen Ulykke md imellem andre Omgivelser . Politiske Forholde bevirkede, at han ved et ulykkeligt Wgtefkab blev knyttet til den smukke , men ligefrem liderlige Margaretha af Valois , og i de nermeft fplgende Aar ester Bartbolomeusnatten levede han ved Hoffet et Liv fuldt af Forlystelser og Udsvcevelser , som vare ner ved ganske at forderve hans indre Menneske . Vi vide fra det foregaaende , hvorledes han 1576 ved Flugten reddede sig fra denne Fornedrelse og forenede sig med sine gamle Venner , Hugenotterne , hvis egentlige Hsvding han nu blev . Frankriges Skjebne forenedes siden paa en <b>vis</b> Maade med hans i flere Decennier , og vi have allerede seet , hvorledes han ikke blot ved Feltherredygtighed , men ogsaa ved sin forsonlige og kloge Politik reddede sin egen Krone og Fedrelandets Ro , maaske endog ligefrem dets Tilverelse .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013100708190'target=_>Funcke, Otto, 1877, Apostelen Paulus til Søs og til Lands</a><h4>236</h4>Man har ofte opkastet det SpMgsmaal , hvad det egentlig er , der danner og grunder Venskabet ? Men hvem vilde tunne sige eller forklare det ? Der maa uden Tvivl vcere et vist Nat ur slægtskab , en umiddelbar Tiltrækningskraft tilstede , der drager det ene Hjcerte mcegtig til det andet ! men felve dette Slcegtskab er en Hemmelighed, og end ikke de største Menneskekendere vilde kunne opdage det . Hvorfor netop disse to Hjcerter slutter sig sammen ( som der staar strevet : „ Da ' blev Jonathans Sjcel bunden til Davids Sjcel , og Jonathan elskede ham som sin egen Sjcel " ) , det er i mange Tilfcelde übegribeligt for enhver tredie Mand . Man hsrer ofte sige om Mgtefolk- „ Det er mig en Gaade , hvorledes den Mand har saaet den <b>Kone</b> , og hvordan de kommer ud af det med hinanden , " og dog kommer de ikke blot ud af det med hinanden , men endogsaa ganske fortrceffelig . Ligeledes kan enhver tredie Mand ofte ikke fatte , hvordan de eller de to unge <b>Piger</b> er blevne Veninder , og dog ligger det klart for Dagen , at de ikke lan vcere borte fra hinanden, ja at de i højeste Grad bidrager til at Mre Livet rigt og lykkeligt for hinanden . Her er en forunderlig Magnetisme , et aandeligt Instinkt hos Sjcelene , der felv bedst fpler , hvor de finder det , der kan udfylde dem . Men til denne umiddelbare TiltrcetNing maa der dog komme endnu et andet vigtigt Element , hvis Venskabet skal have sit rette Materiale , vi vil ikke saa meget sige en <b>vis</b> Overensstemmelse i <b>Dannelse</b> skjo ' nt den uden Tvivl har stor Betydning , men vigtigere endnu er en <b>vis</b> Overensstemmelse i Interesser .

<h4>368</h4>men i Virkeligheden er det meget prosaist , ja vcerre end prosaist : thi Søfolk maa den meste Tid eller bestandig undvcere Familielivets cedle eller forædlende Glceder og Optugtelse . De lever fjoernt fra den milde Indflydelse , <b>Kvinderne</b> udover , ncesten altid mellem sig selv og er alle sammen Mcend i deres bedste Aar , fulde af Overmod i Tillid til deres legemlige Krcrfter og i uafbrudt Kamp med de troløse Elementer . De Midler til <b>Uddannelse</b> , der staar aabne for enhver anden , er de for Drste Delen afstaarne fra , og medens Søfolk og Soldater nu til Dags tager med paa Ss " en eller i Felten , medens de kan vexle Breve med deres Slcegt og Venner , saa var der i gamle Dage slet ikke Tale om sligt . I deres Folks fcelles religiøse Liv kunde de heller ikke tage synderlig Del . Hvad Under da , om de i deres ensformige , gloedelFse Liv snart deres Nydelser paa Veje , hvorover Haarene rejser sig paa vore Hoveder ? Thi ethvert Menneskehjerte tragter efter Gloede ; kan det ikke naa den paa en Maade , der er et Menneske vcerdig , ftger det den paa dyrisk <b>Vis</b> . Det er sørgeligt ; men det er naturligt . Vi antydede nylig , at Søfolk nu til Dags er bedre stillede i mange Henseender ; men alligevel er de meget udsatte for Forforelse , navnlig i fremmede Havne , hvor de er langt borte fra ethvert vaagende H2je og formanende Ord og derimod omgivne af lutter Fristelser til sanselig Lyst . Stader Du derfor paa fromme , scedelig alvorlige Ssfolk , ( hvorpaa der , Gud ste Lov , heller ikke er Mangel,) saa maa Du vcere dobbelt imod dem og tage din Hat dybt af for dem ; thi de har vceret langt vanskeligere stillede end Du . Men trcesser Du paa raa Sofolk , saa lad ikke Farisæeren vaagne i Dig ; Fariscreren tvcert imod druknes i Havet , hvor det er allerdybest , og overgiv ham i det øjemed kun strax til Vandrotterne . Men Du selv stal tcenke : „ I stakkels Fyre ! Hvem veed , hvad jeg var bleven til , hvis jeg i tyve Aar havde stukket i en beget TrFje ? " Og derncest stal den christelige Kjwrlighed overveje , hvad der kan gjeres for at vinde disse Mennesker for Gloeden og Freden i Christendommen .

<h4>538</h4>Indre Respekt for en ydmyg og modig Bekjendelse , selv naar de ler ad den som ad „ forceldet Overtro " . Verden tror ikke , at vor Sjcel er gjennemtrcrngt af Evangeliets Herlighed , af Visheden om det evige Livs Herlighed og om at vcere Guds Born i Christo Jesu , naar vi enten flet ikke kommer frem dermed eller gjM det saa spagfcerdig , medens de dog med en saadan Iver roser og kolporterer deres <b>Visdom</b> og deres Varer , medens de finder paa tufende og atter tufende tilladelige og utilladelige Maader at vinde Menneskene paa og utrcettelige stadig kommer tilbage til deres egne Sager uden at bryde sig om nogen som helst Spot eller Modstand . F . Ex . tomle man tun paa , hvor opfindsomme , ufortrodne og uforskammede visse Lotterikollektdrer er for at fan deres Lodder afsatte . Hvis de ikke havde saa stort Held med deres Uforskammethed , og hvis de ikke ved den ødelagde saa mange Familiefædre , vilde det vcere lystigt at fortcelle om , hvordan do endogfaa sender deres Lodder til os Prcester , med Sirenestemmer priser den salige Fremtid , som den ncesten utvivlsomme Gevinst vil bringe os , indlcegger frankerede Konvolutter til Svar , stadig paa ny sender deres Lytte forjcettende Papirsapostle , hvor ofte man end allerede har fordømt dem til Baal og Brand o . s . v. o . f . v. Saa ivrige , faa tloge , saa utrættelige er Verdens og er det for forfcrngelige eller endog for slette Ting . Men de Ehristne , som nu virkelig har trukket det store Lod , der gjor salig og rig for hele Evigheden, de der kunde <b>vise</b> deres Medmennesker , hvorledes ogsaa de kunde vinde det , og det „ gratis " , disse Christi Disciple har ikke Mod nok ; de har for lidt Mod , og de har for lidt Nidkjcrrhed , fordi de har for lidt Kjcerlighed, fordi de i Virkeligheden ikke er gjennemtrcengte nok af det Menneskes Usalighed , der ikke hMr Jesus til , og af Saligheden i ham . For nogle Dage siden gik jeg forbi en Plads , hvor der var opstablet en stor Mcengde Bygningstommer . Nogle fmaa <b>Piger</b> spillede Bold her , og jeg stod en Stund og morede mig med at se paa dem . Da fis ' s Bolden ind imellem nogle svcere Bjcelter . Man lunde se den ligge

<h4>717</h4>siende med Livsens Vand : han har Vederkvægelser og stille Glceder for dem , hvorom intet Menneske har nogen Anelse , han kan vcekle himmelsk Haab af hver cn Solstraale og af Fuglenes Sang , han lader dem ftle sin faliggjorcnde Ncervcerelse saaledes , at Legeme og Sjcel bliver glade ! ham , den levende Gud . Det har ogsaa St . Paulus erfaret paa Maltas ensomme Klippekyst , hvor han maatte tilbringe tre hele Maaneder . Det var jo sucert nok , Hans Sjcel higede efter Rom , <b>Maalet</b> for hans Rejfe . O , hvor lcengtes ikke hans efter de christne Brødres Forsamling , hvor lcengtes han ikte efter endelig engang at faa en Afgjorelse pllll sin Proces og at faa fin Frihed igjen ! Hvor længtes han ikke efter engang igjen fra Roms Hoje at kaste et Blik vidt ud over hele Verdens Ldb og Guds Riges Tilstand ! Og nu at sidde saaledes til ingeu Nytte og uden <b>Maal</b> og Med hele Vinteren igjennem paa denne N der beskylledes af et stormfuldt Hav ! Men nej , „ til ingen Nytte oa uden <b>Maal</b> og Med " , saaledes tcenkte Paulus ikke . Hau troede for vist og var dybt gjennemtrcengt af Overbevisningen om , at vi ingensteds er til Unytte og uden <b>Maal</b> og Med , hvor Guds Haand ftrer os hen , men at vi overalt bande tan og skal Icrre noget godt og g jo " r s noget godt . Tro Du ogsaa dMe , Du min kjecre Embedsbroder , der sidder ensom og uden Samfund med andre i en eller anden afsides liggende Landsby paa Heden og ofte mener , at Du ml gaa fuldstændig til Grunde der . Tro Du ogsaa det , Du ensomme unge <b>Pige</b> , der maa tjene dit som Gouvernante hos en kold fornem Familie og foler Dig faa forladt fordi herskab og Tyende lader Dig staa alene . Tro Du det ogsaa , Du kjcere troende Soldat , der staar a.ene med din Tro i den hele Kaserne , Du , hvem ingen forstaar, og hvem mange spotter ! Der er endnu 2 ' / . Aar , til atter faar Lov at lomme hjem . Lad ikke om Ulytte overvinde Dig , haab paa Gud , faa ml Du sejre ! grib tit hvor der gives Dig Lejlighed til at yde Hl celp eller <b>vise</b> Kjcerlighed , saa vil selv dine Fjender blive tilfredse

<h4>1077</h4>hpjere , for at bringe dit Hjoerte noermere til ham er denne lille Bog skreven , og hvis den ikke har naaet dette <b>Maal</b> , saa er den til ingen Nytte for Dig . Den er ikke fkreven for at forherlige noget Menneske , ikke engang for at forherlige Paulus , men kun for at <b>vise</b> , hvad det vil sige , at Jesu Krast er mcegtig i din Svaghed , og hvad han endog allerede her paa Jorden kan bringe ud af et stakkels syndigt Menneske . Besind Dig ret paa , hvorledes han stedse vandrede ved din Side , og hvorledes han i gode og onde Dage var dit Livs lifligste Lys . Naar vi tomler os tilbage til vor tidligste Barndom , til de Dage , da en trofast <b>Moder</b> , der nu maaske alt for lcenge siden er gaaet bort , maatte boere os paa sine Arme under det glimrende luletrce , fordi vi ellers var for smaa til at kunne se det rigt dcekkede Festbord , da hun betragtede Dig med straalende Blik , trykkede Dig til sit Hjcerte og sagde : „ Se , mit elskede Barn , alt dette har Christusbarnet bragt Dig , " ikke sandt , da havde vi en Anelse om den salige Forening af Himlens og Jordens Glceder i Julens Konge . I denne vor Frelser kom Gud os saa noer , den Gud , der f , sr sorekom os saa hemmelighedsfuld og fjcern . Han viste sig som et Barn for os VMn . Det hemmelighedsfulde Christusbarn , hvis kjcerlige Hvisken vi havde HM i flere Uger , hvis Vingeslag og Hilsner havde ledsaget os endog i vore Dramme , var Sindbilledet for os paa al guddommelig og menneskelig Elskelighed . Han gjaldt for os som Trøsteren for al jordifk Sorg , da vi nceppe vidste , hvad jordisk Sorg var . Det faldt os saa let at tro , at dette lille Barn , der allerede nu gjorde os saa lykkelige , ogsaa for Fremtiden vilde give os alt godt og tage alt sørgeligt fra os . Og om vi end siden mange Gange fornam noget af den uhyre Smerte , der gjennembcever Verden , saa faldt det os dog strår igjen let at tro : Christusbarnet gjM alt godt , naar vi blot er lydige . Og dette Ideal blev ved at vcere os tro , medens s « a mange andre Idealer viste sig for vor Sjcel og atter forsvandt . At saa sin egen Dukke var den lille <b>Piges</b> , en Kjcephest den lille Drengs HMte Onske .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014072808066'target=_>Flood, Johs., 1880, Jesus alene</a><h4>433</h4>Og Joseph og hans <b>Moder</b> forundrede sig over de Ting , som blcvc sagte om ham . Og Simon velsignede dem og sagde til haus <b>Moder</b> Maria : Se , denne er sat Mange i Israel til Fald og ( til ) Oprcisning og til et Tegn , som imodsigcs , ogsaa din egen <b>Sjel</b> stal et Svcerd gjennemtramge , saa at mange Hjcerters Tanker skulle aabcnbarcs . Og der var en Prophctinde, Anna , Phanuels Datter , af Asers Stamme ; hun var meget gammel og havde levet syv Aar med sin Mand efter sin lomfrustaud ; og hun var uu en Enke ved fire og firsindstyve Aar , som ikke veg fra Templet , tjenende Gnd med Fasten og Beden Nat og Dag . Og hun traadte til i den samme Stnnd og iligcmaade prisede Herren og talede om ham til Alle , som forventede Forlosning i Jerusalem . Og der de havde fuldkommet alle Ting efter Herrens Lov , droge de til Galilea igjen , til deres Stad Nazareth . Men Barnet voxede og blev stcerk i Aanden , fuld af <b>Visdom</b> ; og Guds Naadc var over ham .

<h4>448</h4>modsat af Farisceernes Kulde mod Jesus , belyst af Jesu Ord , se Kananceerindens stcerke Tro , der afvinder ham deu Hjcelp , heudes navnlos ulykkelige Datter endelig behover . Se endelig paa de 3 Kors paa Golgata , hvad er mere ligt hinanden i ydre Kaar , og hvad dog mere forskjelligt i indre Kaar . For den ene Rover bliver Jesus eu Paradisets Tor , for den anden en Spottens Skydeskive . Kjcere Ven , hvad vil Jesus bringe for Dagen af dit Hjcerte ? Vil du stride mod hans Naades Indtryk og Kald . eller vil du ydmygt bore dig for hans Aands Tngt og lade alle Synderes Frelser blive dig dog mest fornoden , fordi din anklagende Samvittighed vifer dig , at du er den stprfte Synder . Da er Jesus sat til et Fald for din Hovmod , som dog kan have en salig Ovreisning tilfolge for dig selv ; thi der er Fald til Ovreisning , ligesom der er Fald til evig Fordærvelse . O der findes for <b>Sjelen</b> saamange Farer og Hindringer , der er saa mange Fald og Stod selv for den Oprigtige paa den trange Vei , at <b>Sjelen</b> ofte bange svorger : Ak hvorledes kan jeg komme frem til <b>Maalet</b> og bevares ? Blir uogeu bevaret ? Da , Veuuer , gjor det godt at se til slige gamle som Anna . Hun er en <b>Kvinde</b> , begavet med profetisk <b>Aand</b> af denHelligaand, ligesom Mose Soster Maria ( 2 Mos . 15 , 20 ) , Deb ora ( Dom . 4 , 4 ) , Hanna ( 1 Sam . 2,1 ) , Hulda ( 2 Kong . 22 , 14 ) , Filippi Dottre ( Apostl . Gjern . 21,8 ) , efter Joels Ord , at Herren vil ndgyde fin <b>Aand</b> over S o nn e r og Dottre ( Joel 3 , 1 ) . Hnn er af de Israeliter, som Esaias spaar om ( 11 , 13 ) : at de 10 Stammer atter sknlde vende sig til Inda Scepter . Hun er hoit bedaget i Tidens Skolegang , med en saare hcederlig Enkestand, som et ret Forbillede paa en kristelig Enke ( 1 Tim . 5 , 5 ) : der scrtter sit Ha ab til Gud og bliver ved i ydmyge Paakaldelser Nat og Dag . Huu var af de crgte Davids Dotre , der bad med Stamfaderen ( Salme 27,4 ) : Et har jeg begjæret af Herren , at jeg maa bo i Herrens Hus alle mit Livs Dage . Selv iden hoie Alderdom , hvor hun saa godt knnde behpve al Kraft , faldt Fasten hende ikke besvcerlig . Hnn lod sig ikke ved nogen hnslig

<h4>567</h4>Landets Hjcerte , saa Sceptret , Hersterstaveu holdt paa at vige fra luda og fra hans Fodder , hvorfor Fredsfyrsten maatte vcere ncer ( 1 Mof . 49. 10 ) , men o gfaa efter samtidige Forfatteres ( losefus og Svetonins ) Beretning blandt de mere dannede hedenske Folk , der havde indseet Afgndstilbedelsens Tomhed og spurgte med Pilatus : Hvad er Sandhed ? En vceldig Lcengsel maa have opfyldt de <b>Vise</b> , siden de begive sig nd paa en saa lang , besvcerlig og bekostelig Reise , som de gjore . Denne Lcengsel er ikke uforklarlig . Den er det tomme Hjcertes Nodsstrig , der for den tomlende cerlige <b>Sjel</b> maa <b>vise</b> ham , at han endnu ikke har fundet den sande Fred og Glcede . Den samme Lcengsel trceffer saa ofte Misfjoncererne end den Dag idag ude blaudt Hedningerne . Den er for Misfjoncererne en herlig Prcediken omHerrensunderlige Naade : Gud s mangfoldige <b>Visdom</b> i Menigheden til at vinde Sjele for sit Rige , og det giver dem stedse fornyet Haab om Guds Riges Sags Seier , hvor morkt det stundom end kan se ud . Vi hpre f . Er . om eu Hindu <b>pige</b> fra det nordlige Ostindien , der var romt fra sine Slcegtninge og blev solgt til Slavinde hos en rig Muhamedanerinde . Pigen havde truffet eu kjcerlig <b>Kvinde</b> , der fattede Godhed fom en <b>Moder</b> for hende og opdrog og underviste hende i Muhamedanernes Bibel , Koranen . Hun levede i Herlighed og Glcede , men med Et faldt en dyb Sorgmodighed over den Unge . Moderen , som hun kaldtes , forsogte ved Adspredelser , Reiser , Lystigheder at opmuntre den Unge , men det hjalp ikke , hun klagede over sin Synd . Muhamedanske Prcester bleve tilkaldte, de forelceste hende Nonner , alt hjalp dog intet . Hun vendte sig til Hindnernes Prcester og disse , der kaldes Braminer, bad hende at ofre hver Dag et Blomsteroffer og hver anden Uge en Buk til de oude Aander , der cengftede hende . Men det hjalp heller ikke noget . Hjcertet sukkede under Ufredens Byrde . Da trceder en fattig Betler ind , faar en Almisse og omtaler tilfccldigvis en gavmild Mand , som forfikrer om en Forlosning , fom en <b>vis</b> lefus har gjort . End stiger et Haab i hendes <b>Sjel</b> om at faa Hjcelp , hun iler til denne Mand , der er en kristen Missjoncer , horer om Jesus

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016100428003'target=_>Andresen, K.N., 1846, Om Afskaffelse af Brændevinets Brug som Drik, eller Afholdsreformen og Midlerne til denne Reforms Gjennemførelse</a><h4>370</h4>med Begjærlighed sluges af Oldinger , Mand , <b>Kvinder</b> og Bom , der ligesom danner Middelpunktet for de simplere Klassers Dnster , Planer og Forhaabninger . Det er dernæst en aldeles urigtig Forestilling at tænke sig Brcendedeviinsoverdaadigheden hos de simplere Klasser som beslægtet med og ligesom en mod lavere <b>Formaal</b> rettet Ittring af den blandt de ovrige Stænder ikke sjelden stedfindende Tilboiettghed til Lmus . Den store Forstjel er nemlig den , at der her næsten altid idetmindste til en <b>vis</b> Grad finder en Beregning Sted over , hvad egne okonomiste Kaar tilstede at indlade sig paa eller ikke , ligesom overhovedet Lmus i een Retning her idetmindste i Regelen tilfredsstilles ved Besparelser i elt anden , medens derimod blandt de saakaldte simplere Stænder en saadanßeregning i Reglen slet M anstilles ; den simple Arbeider sporger slet ikke , hvormeget Brcendeviin han har Naad til at drikke , men kun , hvormeget der horer til for ham i hans Stilling; den , der fortjener Lidet eller Intet om Dagen , vil have ligesaa meget som den , der har god Fortjeneste ; med andre Ord : Brcendevinet er det Forste og det Sidste , sor dette Behov maa der først og fremst sorgcs ; bliver der Noget tilbage , faae <b>Kone</b> og Born sin Deel , i modsat Fald Intet ; thi Manden har selv „ ikke tjent Mere end til Brcendeviin om Dagen . " Her er ikke Tale om Lmus , men om et ligefremt Raseri , en fuldstændig Begrebsforvirring, hvorved Brcendevinet i det daglige Livs Fcerd kommer til at spille Hovedrollen i Livet blandt den lavere Folkeklasse , indtil det tilsidst — ikke sjelden — kommer dertil , at Mand , <b>Kone</b> og Born , kort hele Familien , saagodtsom „ lever af bare Brcendeviin . " Dertil bringer Brcendeviinsdrikken det , og Enhver seer lettelig den himmelvide Forstjel mellem en saadan Tingenes Tilstand og den med de gjærede Drikkes Brug forbundne Ulempe , at

<h4>703</h4>trcengt , fordi man ikke uden en fuldkommen Afskaffelse af Brugen kan forebygge Misbrugen med dens odelæggende Virkninger: folger det af sig selv , at Planen kun derved kan lykkes , at der fra alle Kanter udtaler sig en Misbilligelse med Brugen selv og et almindeligt Onste om dens frivillige Ophor , hvilket igjen kun kan stee ved at indtræde i en Afholdsforening . Thi om Den , der ikke er Medlem af en Afholdsforening , kan man naturligviis ikke vide af hvad Mening han ( eller hun ) er , hvorimod der i Indtrædelsen i en Afholdsforening fra Vedkommendes Side ligger en bestemt udtalt Erklæring om , at man anseer et almindeligt Ofthor med al Brcendeviinsdrik for paa Grund af Omstændighederne onsteligt . Den Opinion , der saaledes udtaler sig gjennem Afholdsforeningerne og Indtrædelse i disse , er derfor ene istand til at bevirke , hvad der her stal bevirkes . Man er nemlig ialmindelighed naturligviis til en <b>vis</b> Grad ligegyldig decls for Ondet selv deels for det i den nyere Tid opstillede Middel mod samme , det Forste formedelst Vanens Indflydelse , det Sidste fordi man ikke synderlig tænker derover iscer i en Tid , da der er saa Mangt og Meget igjære ; til i denne Henseende at vække Eftertanken og Interessen hos Andre for det hele vigtige Anliggende er derfor den personlige Indtrædelse ei alene det kraftigste men endog det eneste tilstrekkelige Middel der gives . Det er derfor saa langt fra , at Nogen ( Mand eller <b>Kvinde</b> ) ved at indtræde i en Afholdsforening vækker hos Andre Tanken om , at man selv stod i Fare for at bukke under sor Brcendevinets Forforelser , at man tvertimod netop ved sin Indtrædelse lægger sin Nvillie mod Brcendevinets Brug som Drik ( selv i ringeste <b>Maal</b> ) klart for Dagen . Og dette er den store Fordeel , som opnaaes ved den Enkeltes Indtrædelse . Mangen , som for var lunken og ligegyldig og ei havde tænkt videre over Sagen , bliver

<h4>832</h4>selv klart opfatter sit Væsen som en Foranstaltning ei mod Fyldenet men mod selve Brcendevinets Brug som Drik i ringeste <b>Maal</b> , altsaa ei mod den umaadelige men mod den maadeholdne Nydelse . At bringe den maadeholdne Brug af Brcendeviin og deraf tillavede Drikke til lidt efter lidt at ophore , er Afholdsforeningernes eneste <b>Formaal</b> . Dette kan igjen alene bevirkes derved , at efterhaanden Flere og Flere netop iblandt de allerede iforveien Afholdne eller idetmindste Wdruelige indtræde i Foreningen ; thi Indtrædelsens Hensigt er ei at sikkre sig selv mod en befrygtet Fare i denne Henseende , men alene den , offentlig at udtale sin Mening i denne Sag , d . e . offentlig at lccgge sin Misbilligelse med Brcendevinets Brug som Drik navnlig hos den opvorende Slccgt for Dagen . Som en Følge heraf maa enhver Forening i kraftigste Maade protestere mod den endnu fast ligesaa almindelige som grundfalske Mening , at det fornemmelig er drikfceldige Personer , hvis Indtrædelse snstes . Da Erfaring viser , at hverken Skrift eller Tale formaae at adsprede den Taage , der endnu hviler over den hele Reform i denne Henseende , er der intet andet Middel tilbage for Foreningerne end i Gjerningen at <b>vise</b> , at Tingen ikke er at førstaae paa denne Maade . Man afvise derfor stedse og ved enhver Leilighed ( gjcnnem Foredrag i Foreningens offentlige <b>Moder</b> saavelsom ved private Anledninger , hvor Nodvendigheden byder det ) det gamle grundlose Snak om „ det Dnflelige i , at Den eller Den Omar derved sigtes til en bekjendt drikfceldig Person ) maatte indtræde " o . s . v. Det er i Sandhed ikke lidet nu-erkeligt , hvorledes henne Mlsforstaaelse af Sagen i saa lang Tid haardnakkct har trodset enhver Vestrcebelse for at hceve den . Kommer en Person ravende ned ad Gaden eller Landcveicn , eller ligger han allerede paa sine Gjermnaer,

<h4>852</h4>tordne og tale mod Kjøb og Salg , Brccndmg og Traktere « hos Andre o . s . v. Man stoder saaledes ikke sjelden paa Mænd , som ganfle og aldeles billige den hele Foranstaltning og gjerne vilde bidrage , hvad de kunde , ttl Diemedets Fremme . Med alt dette seer man alligevel , at de ikkedestomindre ikke indtræde . Sporger man dem om Aarsagen hertil , faaer man altid det samme Svar : " Jeg skulde ' gjerne indtræde , da jeg alligevel ikke drikker Brcendeviin, dersom jeg kunde virke noget for Sagen . " Dette " Virke " er et ganste besynderligt Ord i Afholdsanliggendet ; man horer det meget hyppig ved forskjellige Anledninger , hvor dette Anliggende omtales , og dog er det vist , at det i den Betydning , hvori man ved slige Leiligheder gjerne førstaaer det , allermindst horer hjemme i dette Anliggende ; thi Sagenstaaervirkelig saaledes : der erlntet at gjore , at sige for Mccngden af Foreningernes Medlemmer . Det saagodtsom Eneste , der er at foretage , bestaaer i ved Skrift oq Tale at bidrage til at fremme vedkommende Forcmngs <b>Formaal</b> , hvoraf man er Medlem ; det Forste kan ikke vel tankes at falde uden i Enkeltes Lod , da i modsat Fald Verden vilde blive altfor fuld af Afholdsskrifter ( hvilket den i en <b>vis</b> Henseende maastee allerede er ) ; det Sidste , i Foreningernes offentlige <b>Moder</b> at holde Taler eller Foredrag, er naturligviis Noget , man ikke kan forlange af Enhver iscer , da man i modsat Fald ogsaa i denne Henseende fik Formeget af det Gode . Den Tale om " at virke for Sagen " , førsaavidt det ikke bestaaer i een af eller begge de ncevnte Virkemaader , har nccrmest sin Grund i den samme urigtige Forestilling , der fremkalder saagodtsom alle Misforståelser i denne Sag , nemlig at det her gaaer los paa at bidrage til Brcendeviinsdrikkens Formindskelse eller Indskrænkning . Men det er netop denne man som Medlem af en Afholdsforening siet ikke har med at bestille .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010052706060'target=_>Welhaven, J.S., 1867, Samlede Skrifter</a><h4>318</h4>Se , Kummets bugnende Bug ( ! ) . 0 Hvo besvangred ? — Ohebiel henviser ham til Guds <b>Visdom</b> og ordnende Kraft . Messias fremtræder som Guds Kepræsentant ; paa hans Bud opstiger Kajahel , Livets <b>Aand</b> , og Obaddon , Tilintetgjørelsens <b>Aand</b> , af Jorden . Kajahel fremkalder om Dagen blide Livs-Former . Obaddon søger at modarbeide denne Skaben ved af Nattens Skjød at udvikle vilde , ødelæggende Skikkelser . Kajahel er <b>Kvinde</b> , Obaddon Mand . De udgjøre til — sammen et uenigt Ægtepar :

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008092303002'target=_>Brandt, Fr., 1855, Samling af mærkelige Højesteretsdomme i Tidsrummet 1815-1835</a><h4>2628</h4>den folgende Formiddag , uden at Barnet nogctstcds var at finde . , I sin Solg herover bestuttede Foreldrene , ester flere af de Tilstedeverendes Naad , at henvende sig til Angjeldende , der stod i Ny for at vere en saakaldet " klog <b>Kone</b> , " som forstod sig paa Gjenvisning , Signen m . v. , og afsendte til den Ende et Bud med 24 si som Gave til Angj . Ved Modtagelsen heraf lovede Angj . , " at hun nok , hvis Gud vilde , skulde flasse Barnet igjen , " og forfojede sig derefter ind i et Sideverelst ( " sit Lonkammcr " ) med et det borteblevne Barn tilhsrende og af Budet medbragt Kledningsstykke , hvilket hun foregav , at hun nodvendigen maatte have , for at lcrgge ved sit " Verk , " hvis hun stulde kunne give ham nogen Underretning om Barnets OpholdZsted . Dette saakaldte " Verk " stulde , ester Angjeldendes Foregivende , have bcstaaet i en af en stor Mengde Fjedre sanimensat . i en Kiste i Angjcrldendes Lonkammer staaende Mastine , der stulde tjene som Organ for et overnaturligt Vesen , med hvem Angj . paa denne Maade stod i Forbindelse , og ville flere af Vidnerne ofte have hort denne Maskine - som aldrig nogen anden Dodelig end Angj . stal , have seetnaar den blev sat i Gang , give en Lyd fra sig , som af en nedgaaende Urvekker. Efter at verre vendt tilbage fra sit Lonkammer , crklerede Angj . , at hun ved Beskuelsen af sit Verk var kommet til <b>Kundskab</b> om , at Barnet var indtaget i Jorden af de Underjordiske , der beherskede 2 i Nerheden af Foreldrenes Bolig beliggende og af hende betegnede Hoje ; at det sammesteds havde det meget godt og paa ingen Maade lengtes ester Hjemmet , at det ikke kunde nytte , om nok saa mange Mennesker sogte efter det , men at Foreldrene , hvis de vilde have Barnet igjen , stulde ophore med al Eftersogning , og derimod stille Vagt omkring bemeldte Hoje og ved sin Husdsr , og stulde disse Vegtere alle vere konfirmerede . Mennesker , hvorhos de ved Husdoren udsatte skulde have godt Staal og Solv hos sig , ligesom de ved Hojen opstillede , naar de vilde bevege sig , stulde gaa omkring Hojene imod Solen . Naar disse Forskrifter iagttoges , forsikrede hun , at Barnet , uden at hun dog kunde bestemme naar , vilde af sig selv komme tilbage , og stulde Vagten ved Doren , naar den saa Barnet komme , kaste Staalet over dets Hoved , hvorved de Underjordiske vilde tabe sin Magt over det . I Nvrigt lagde hun dog til , at det var godt at lyse ester Barnet ved Kirkebakken . Ester Budets Udsigende stal Angj . ogsaa have " ytret , at hun engang forhen havde skaffet et borteblcvet Barn tilveje igjen , men at hendes Verk ved denne Lcjlighcd var gaact istykker , saa at hun havde maattet anvende flere Dalers Bekostning for at faa det i Stand igjen . I Tillit til Angjeldendes Erklering gik de , der havde segt efter Barnet , hver til Sit , hvorimod Foreldrene strar gav sig iferd med paa det Nojagtigste at efterkomme enhver af Angjeldendes Forskrifter . Ester forgjeves at have ventet i nogle Timer paa , at Barnet stulde komme tilbage , gik Faderen ' selv til Angjeldende , og bad hende paa det Indstendigste om ikke , at narre ham , da han var <b>vis</b> paa , at dersom Barnet blot fandtes over Jorden , saa kunde og vilde alle Bygdens Folk vere ham behjelpelig med at lede ester det . Angj . vedblev imidlertid , ester paa ny at have beskuet " sit

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2007081503001'target=_>Munch, P.A., 1859, Det norske Folks Historie</a><h4>1279</h4>paa sit Sted ncrrmere stal berettes . Hun paastod , at Fru Ingcbjorg Erlingsdatter havde solflt bende ' ) , det vil vel sige for Penge ladet sig bevcrge til at overgive hende til Folk , der snstedexhende af Vejen , og at disse igjen havde skaffet bende bort til andre Lande . Heraf fremgaar i alle Fald saa meget , at Frulngebjorg , saaledes som det ogfaa ovenfor er antaget , maa have vcrret med paa Rejsen til Orknorrne , og haft Varnet under sin Varetcrgt , samt at der rimeligviis maa have vcrret noget i den Maade , bvorpaa hun under Sygdommen behandlede bende , der var hendes ovrige Omgivelser paafaldende og gav Anledning til mistænkelige Rygter . Usandsynligt er det vistnok ikke at en eller anden af de Prætendenter til Skotlands Krone , som meldte sig efter Margretes 3 od , og som Maa meente at kun dette <b>Pigebarn</b> stod imellem bam og bans Onssers <b>Maal</b> , kan have hemmeligt indfundet sig bos Fru Ingebjorg og ved store Pengesummer sogt at bestikke hende til at rydde Barnet af Vejen , eller overlade ham det , men det synes dog ikke rimeligt , at hun skulde bave kunnet bekvemme sig til , eller endog vovet at begaa en saadan Nidingsstreg ; det maatte da ligefrem vcrre af Had til Kongen og Kongehuset over den haarde Behandling , hendes Broder Alf Jarl havde maattet lide , hvis bedrsvrlige Endeligt hun just i den Tid begrcrd . Men saa meget er dog sikkert , at Kong Erik selv , hendes egen Fader , maa . have anseet sig <b>vis</b> paa at hun var dsd , ligesom heller ikke Kong Edward , der dog tabte saa meget ved dette Dsdsfald , tvivlede drrpaa ; heller ikke er der Spor af at Fru Ingebjorgs Mand , Hr. Thore Haakonsson , faldt i nogen Unaade , hvilket dog maastre vilde have vwret Tilfaldet , hvis hans <b>Hustru</b> havde begaaet en saa skammelig Handling ; heller ikke synes hun selv at have vceret under mindste Forfslgelse ; hun dsde ikke forend 1315 , fremdeles , som det lader , nydende den storste Anseelse Troen paa at Margrete eller Pigen fra Norge ( tne UM of som Skoterne kaldte hende , ikke var dsd , men kun bragt til Side , har saaledes vistnok kun hersket blandt de lavere Klasser , fornemmelig i Skotland , hvor de Begivenheder , hvortil hendes Dsd gav Anledning , nok kunde bringe alle Fædrelandsvenner til at snste , at Efterretningen maatte vcere usand . Som bekjendt , optraadte Kong Gdttmrd nu ganske i Egenskab af Skotlands Leensherre , idet han indkaldte de skotske Magnater til et Mode eller Parlement allerede i det folgende Aars Mai Maaned ( 1291 ) , for her at handle om Rigets Anliggender , og navnlig om hvilken der maatte blive at vcelge til Konge af de Prætendenter , sotn havde

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014032438007'target=_>Marie, 1878, I det Stille</a><h4>723</h4>Udbredelse og skaffe mig et anseet Navn i Videnskabens Rige . Jeg har anvendt Aar af mit Liv paa dette Arbeide , ofret det mine Dage og Nætter , og Frugten af denne Flid og Gransken tilfredsstillede mig i rigeligt <b>Maal</b> . Da hørte jeg en Aftenstund en liden <b>Pige</b> forelæse mig noget , som i al sin enkle Storhed lynsnart slog ned i mit <b>Tankeliv</b> , lod mig se Dunkelhed og faml ende Søgen , hvor jeg havde troet at sprede Lys og Forstaaelse , og lærte mig , at min Bog manglede det væsentligste , Livsstrømmen , som skulde give det døde Bogstav belivende Kraft : Den manglede Beaandelse. Og under den stille Indflydelse af det Ord , jeg i hin Stund havde hørt og senere seiv gransket i saa mangen ensom Time , begyndte jeg paany at gjennemlæse og tildels omarbeide Manuskriptet , og hvis Bogen nu vinder den Anerkjendelse , jeg venter , da maa det for en stor Del tilskrives de Tanker , der den Gang vaktes til Live hos mig og senere fik ny Næring , medens jeg , delt inellem Frygt og Haab , vaagede ved dette Barns Sygeleie og derfrå sendte mit Suk til ham , som holder Liv og Død i sin almægtige Haand , indtil jeg endelig kunde udbryde : « Lovet være Herren ; thi han har hørt mine ydmyge Begjæringers Røst » » !

<h4>1254</h4>føiede : « Men saa kom den Tid , da ogsaa jeg blev greben af denne stærke , hemmelighedsfulde Magt , som jeg atter og atter havde fornegtet og forhaanet . Det kom over mig som med Trolddom , og jeg mærkede det ikke , før det alt var for silde . Var det Nemesis " s hævnende Gjengjældelse , der rammede mig , fordi jeg havde vovet at betvivle Tilværelsen af den bedste og reneste Følelse , Menneskehjertet eier ? Eller havde jeg i Dybet af min Sjæl halv übevidst huset et Ideal , som min Fantasi gav Liv og Skikkelse , da mit verdenstrætte Sind kom i Berørelse med hans unge , naturfriske Væsen ? Maaske har der været Øieblikke , da jeg har seet Idealet glide ned fra det Alter , hvorpaa jeg havde stillet det ; men hvilken <b>Kvinde</b> finder ikke Undskyldninger for den Mand , hun elsker ? Jeg lukkede Øinene og forsøgte at indbilde mig , at det Drømmebillede , mit Hjerte havde foregjøglet mig , var Sandhed . Nu er jeg vaagnet , vaagnet for hele Livet ! Gunnar , hvis Du i den sidste Tid har følt min Kjærlighed til Dig som en trykkende Byrde , saa vær <b>vis</b> paa , den kan ikke have været Dig tungere at bære end den , Livet med alle dets smaalige Hensyn og usle <b>Formaal</b> fra dette Øieblik paalægger mig . Jeg tror , vi ere kvit » .

<h4>1719</h4>mange tilvante Former , hvormed det ydre Liv omgjærder al sin Gjøren og Laden i de saakaldte dannede Kredse . Naar jeg undtager vor deltagende Husfælle , Madame Strøm , og maaske Presten Smith og hans <b>Hustru</b> , valdet neppe nogen , der anede den Hovedomvæltning , som havde fundet Sted i Falk " s hele Væsen og Tankesæt , og dog var dette en Begivenhed , der greb ind i hans dybeste Hjerteliv , spredte sine velsignelsesrige Virkninger ud over hans hele Fremtid og forvandlede den mest hverdagslige Gjerning til en Gjenstand af Betydenhed, fordi den nu var bleven et Arbeide for Gud ; og dog sporedes Virkningen af det indtrufne maaske allermest deri , at hans sjælelige Øie mere og mere oplodes for Erkjendelsen af Guds <b>vise</b> og opdragende Hensigter, ligesaa vel , nåar han atter lod Prøvelsens Time oprinde over os , som nåar Glæde og Lykke i rigeligt <b>Maal</b> bleve os til Del .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010042703044'target=_>Paalzow, Henriette, 1845, Jakob van der Nees</a><h4>1639</h4> leg vil aldrig glemme det Blik , hvormed hun maalte miss , da jeg forst traadte hende imsde som Modstandcrindc — ak ! hun maatte anfce mig for en meget foragtelig Meddcilcrindc ; thi ligeover for denne blomst , ende Skjonhed , der betragtcde sin <b>Aand</b> som en vkksom Sminte til at forhoie sinr Zindighcder , stod den blegc , spge <b>Kone</b> , som aldrig havde havt nogen <b>Aand</b> , der aldria , crholdt mcgcn <b>Dannelse</b> . Men lnaaskee var dette uanselige Mrc min og i min Ncrrvcrrelse forlatte hun Elstovsscencrnc med min Mand , maaflee for at vift mig hendcs uindsircrnlede Herrcdomme , for at give mig den Vished , at jeg havde tabt ham . »

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011060704032'target=_>Feragen, A.M., 1875, Hjem og Skole</a><h4>96</h4>derligt paa Hjerte , til Foreldre , der i sine Born ser den dyrebareste Gave og bedste Eiendom , Vorherre har betroet dem , hvis Lyst og Glede det derfor er at bere Omforg for de fmaa i <b>aandelig</b> og legemlig Henfeende , hegne om dem og folge dem med vaagent Vie , faaledes som Kjerligheden tilskynder dem . Saadanne Foreldre vil jeg onste <b>Visdom</b> fra Gud til sin ansvarsfulde Gjerning ; thi det maa I vide , at den naturlige Kjerlighed saa let kan blive misvisende og vildledende og saaledes , uden et klarere Syn paa Sagen , fore paa Afveie . Paa den ene Side kan Barnet forgudes , nåar I lukker Vinene for dets Feil og synes , at alt , hvad det gjor , er godt og vel , opfylder ethvert af dets Dnsier og viser en Foielighed , der giver dets Egensind Nering , saa at det tilslut ender med , hvad der horer til den bagvendte Verden , at Bornene bliver de bydende og Foreldrene de lydende. Paa den anden Side hender det itte saa sjelden , at Fader eller <b>Moder</b> eller begge tilsammen under sit vistnok velmente Strev med at vende Barnets Sind sta det , som ondt er , behandler det med en saa utidig Strenghed , at den mangen Gang grendser til übarmhjertig Haardhed , idet de forvekster barnlig Muuterhed med Udbrud af Ondsiab og straffer begge Dele ligedan . Naar nu Barnet ved Siden Heras forbydes Deltagelfe i alslags uskyldig Leg og Spog , der fremstilles for det forn strafverdige Ting , medens det tilholdes at <b>vise</b> et Alvor , som er det aldeles fremmed , faa itte alene forkues og forkvakles den barnlige <b>Aand</b> , men meddeles endog de mest forvirrede Vegreber om Forskjellen mellem ondt og godt . I ser

<h4>112</h4>for itte . Vi maa komme ihu , at faavel <b>Maal</b> forn Midler for Borneovdragelfen opfattes hoift forstjelligt af de forskjellige Folkeslag og Troessamfund . Joden opdrager saaledes sine Born paa jodist <b>Vis</b> , Hedningen paa hedenst og Muhamedanen paa sit <b>Vis</b> ; altsaa hver efter sine Grundsetninger og sit Syn paa Livet . Vore Forfedre i Hedenold tog fig ogfaa af fine Boms Opdragelfe efter fit Livsfyn . . De vennede nemlig sine Gutteborn tidligt til en hoi Grad af Haardforhed og ovede dem i Udholdenhed , Mod og Tapperhed . <b>Maalet</b> var , at de skulde blive Idretsmend efter den Tids Begreber , nemlig djerve Vikinger og frygtlofe Krigsmend . Midlerne hertil var Legemsovelfer og Lege as forstjelligt og forn oftest voldsomt Slag , men vel stittede til at udvikle Legemskrefterne og frembringe Djervhed , Frygtloshed og Foragt for Fare , ja for felve Doden . Ved de Gudeog Heltesagn , som i Sang og Tale gik fra Fader til Son gjennem Slegterne , opflammedes Gutten tidligt til at ove alle Slags mandige Idretter , medens der paa hans aandelige Udvikling forresten blev lagt liden Vegt . Alt gik ud paa , at han som voksen kunde tåge kraftig Del i Kamplivet og ove Daad , forn var verd at mindes af Efterstegten . Det blev derfor feet som et godt Tegn , nåar Gutten tidligt i Kamp udgjod sin Modstanders Blod . Dotrene blev opdragne af sine <b>Modre</b> og fik itte Lov til , saaledes som Gutterne , at raade sig selv , men var langt mere nnder Tugt . De vennedes til Bluferdighed og Kysthed og ovedes i mange huslige Sysler , saasom at sy , spinde , veve , brygge , båge og lignende . At vere trosast i Venstao , gjestfri

<h4>215</h4>og tilslut foragtede sig selv . At en stig Fremgangsmaade var alt andet end opdragende , ja , at den ligefrem var odeleggende , maa enhver kunne stjonne . Men heldigvis er den nutildags itte i Brug uden som Undtagelfe , og jeg kan derfor fpare mig for en videre Fremstilling af , hvor fordervelig og fordommelig Ven er . Derimod er det en altfor almindelig Skolestil i vor Tid at bruge Barnets LEresfolelfe til et Drivhjul for Flid og Kappelyst , hvilket settes i Gang ved daglig Karaktergivning , idelig Op- og Nedflytning og lignende Pirremidler , hvorved den Uretferdighed begaaes , at den med fmaa Evner , om end villig og flittig , stot og stadigt maa se sig tilbagesat og ydmyget, medens den letnemme , stjont mangen Gang doven og ligegyldig , ser sig ophoiet og rost , fordi han med Lethed kan faa tillivs og gjengive de forefatte Lekser , hvilket hin med al sin Flid og Moie ikte formaar . Hvad virkelig godt der kan komme ud af denne evindelige Karaktergivning ved hver Overhoring , er itte let at forståa og kan vel vanskelig paavises , men hvad ondt denne og al anden kunstig Fremdrivning sorer med sig . ligger klart for Dagen for enhver , forn kjender noget til Barnets Sjeleliv . <b>Maalet</b> for dets Flid forryttes paa en ligesaa sorgelig som latterlig Maade , idet Gutten eller Pigen driver paa med sit Skolearbeide itte saa meget for at famle Kundstab og gjore sig dygtig til at gaa ud i Livet , endnu mindre fordi de elsker Oplysning og mindst maaste af Lydighed mod Fader og <b>Moder</b> , men for af Lereren at faa et 1 - Tal eller dog mindst et 2 - Tal i sin Bog at <b>vise</b> stem og fornoie fig over , faa at vidste de sig trygge

<h4>242</h4>det , som er ondt til det , som er godt , og efterhvert som den . . under Herrens Tugt og Formaning " faar en saa dan Retning , maa den styrkes ved gode og paalidelige Grund setning er . Disse hentes intetsteds sikrere end i Guds Ord , og den overste af dem bliver da den , at ~ vi boie vor Villie under Guds Villie " , thi forst da bliver den i Sandhed fri og kraftig . Alene Guds Villie er god , i sig felv og god imod os ; for kuns ved den kan Mennefleaanden naa fit hoieste <b>Maal</b> og Med , fores frem , loftes op og komme hjem til ham felv , for at „ vere evig glad i Vor Herres Jesu Navn " . Alt det , forn strider imod Guds Villie , drager bare tilbage , gjor ufri , magtstjeler og hindrer os paa Hjenweien , fordi det drager nedad istedetfor op ad . Tenk derfor , om vor Villie kunde gaa op i Guds Villie , hvor vi da blev fri , lykkelige , salige ! Ja , det er netop det , Vorherre vil , og derfor har han jo felv lert os i vort velsignede Fadervor at bede : „ Ske din Villie ! " Ja ler dit Barn , du Fader og <b>Moder</b> , isse bare at bede saaledes , men ler det tidligt at kjende og elske Guds herlige og <b>vise</b> Villie og at boie sig under den og saaledes ogsaa boie sig under eders Villie , nåar denne stemmer med Guds . Det er denne Villiens Boielse , vi kalder Lydighed , og derom vil vi tale lidt udforligere med hverandre .

<h4>599</h4>Hvad jeg herefter har at sige om Maaden at tåge denne Sag vaa , det stal ifer vere sagt til de Fedre og <b>Modre</b> , som har gjort Borneopdragelsen til sit Livs Opgave , og som derfor gjerne vil hore , prove og folge et godt og velment Rand . Det nytter i Almindelighed lidet eller intet at tale til ensidige , fordomsfulde eller ligegyldige Mennesker . Det er deres <b>Vis</b> at forkaste uprovet ethvertßaad og enhver <b>Lerdom</b> , som itte stemmer med deres egne Synsmaader og tilvante Leveset .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009072003023'target=_>Helveg, F, 1862, Spaadommenne, eller Gud i Historien</a><h4>1149</h4>navnlig i Kolossenserbrevet , end Johannes , som har betonet dette „ Sønnens " Forhold til Guds Selvaabenbaring og derfor ogsaa til Skabelse og Skabning . For nu at finde Sammenknytningspunktet af lagttagelsen om Forholdet imellem Elohim og Jehova med hine nytestamentlig dogmatiske Bemærkninger , saa veed jeg ikke rettere at kunne paavise elet end ved at fremhæve Jehovas særegne Stilling til Mennesket ( I , p. 52 ) . Gud er som Skaberen i de seks Dage , ogsaa som den der skaher Mennesket i sit Billede og overgiver ham saa at sige alt det Skabte , at han enten maa herske over det eller det dog selv uden egentlig Herreelømme tjene ham , endnu Elohim . Men saasnart det skal <b>vise</b> sig , at Mennesket har en Bestemmelse udover ' den Stilling , som er ham ligefrem anvist , en Opgave , som gaar udover det Synlige , et <b>Formaal</b> , som først kan naas i Tiden , i en af Mennesket maaske uafhængig Fremtid , saa viser ogsaa Gud sig som Jehova ; dermed ere Gud og Menneske bragte nærmere sammen end Tilfældet var selv ved Skabelsen i Guds Billede fra først af og følgelig er der forjættet , at Fællesskabet imellem Gud og Mennesket trods Begrændsningen i Haven eller netop paa Grund deraf kan og skal blive end et inderligere . Forholdet imellem Gud og Mand har følgelig sin Historie , hvis første Begivenhed er <b>Kvindens</b> Skabelse , den anden Mandens Forførelse ved <b>Kvinde</b> og Slange . Guds Opgave er nu at skille <b>Kvinde</b> fra Slange og atter at gjøre Mand og <b>Kvinde</b> til et ; denne Gjerning kan kun gaa frem gjennem Udviklingen af alle menneskelige Forhold, hvis Grundlag er Mandens Forhold til <b>Kvinden</b> ( en elybere Betragtning af Kunst og Videnskab vil godtgjøre elet sexuale Forhold som uomstødelig Naturbasis for alt det Aandelige i Mennesket , der skal udvikles ) , saavelsom gjennem Udvikling af Forholdet til Gud , Religionens Historie. Inden Syndefaldet var det tilstrækkeligt til Guds fulde Tilnærmelse til Mennesket , at han fremtraadte som Jehova Elohim — Spændingen bestod end ikke imellem

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009072003024'target=_>Lassen, Hartvig, 1866, Henrik Wergeland og hans samtid</a><h4>2220</h4>men aldrig har vel nogen begavet <b>Aand</b> brugt dem mere samvittighedsløst , og derfor har han et bittert Regnskab at aflægge . Det første han gjorde , da han havde tåget sin Skat i Besiddelse , var at grave den ned i Jorden ; han troede , at det var den retteste Maade at gjøde Marken med Guid , det er i det Mindste vist , at man næsten aldrig har seet nogen bølgende Grøde paa hans Ågre , men vel undertiden glimtende Guldstykker i Mulden . De Kræfter , der strax lode sig tilsyne , var en voldsom Lidenskabelighed , en urolig Phantasi tdligemed en høist bevægelig Ideeassociation , og en Rigdom af løse Ideer i Forbindelse med en paafaldende Styrke og Pragt i den enkelte Tanke og i dens Udtryk . Alt dette var jo godt nok , men Ulykken var , at en overordentlig Umodenhed ogsaa lod sig tilsyne , og denne har fra Begyndelsen fordærvet Alt . Da Ægeus drog bort fra sin <b>Hustru</b> , lagde han sit Sværd under en Steen , og befalede , at først , nåar hans Søn kunde løfte denne , maatte han tåge Sværdet i Brug . Men derfor var ogsaa Theseus saa stærk da han kom til Kreta , at han kunde dræbe Minotauros . Af Wergelands Idrætter kan man see , at han ingen Steen har løftet . Det er tvertimod synligt af alle hans Værker , ligefra Farcen „ Ah " og de første Oder i Studentersamfundets Blad til hans Skrifter fra dette Aar , at han har begyndt sin Digterbane , forinden han endnu havde Begreb om hvad Poesi egentlig var for en Ting . Hans Lidenskabebghed fandt endnu ingen Modvægt i hans Forstand ; han havde endnu ikke dannet sig noget sammenhængende Livsbillede, og ikke tilegnet sig nogen klar gjennemlevet Anskuelse af nogen Ting ; han havde følgelig Intet at give , fordi han selv Intet eiede . Men Wergeland var uvidende om , at Digteren har Pligter mod sit Talent . En Harpe er en Gave , og Digterens eget Bryst maa være Harpensßesonnancebund; men derfor maa ogsaa hans Bryst være fuldkommen styrket og renset; thi i denne Resonnancebund maa alle de Toner , der skulle udgaae gjennem Strengene , allerede i Forveien ligge tause og slumrende . Det er en uhyre Vildfarelse , hvortil Wergeland har været hengiven , at man kun behøvede Digteranlæg for tidige at være Digter , og at man da strax kunde begynde at improvisere uden at have hørt og seet og tænkt Noget , kort uden Studium . Man venter dog ikke af Nogen , at han skal kunne male en Fos eder et Palmetræ , nåar han aldrig har seet disse Gjenstande eller hørt Noget om dem , eller at Nogen a priori skal vide , hvorledes man gjør Uhre , eller forstaae Fransk ; men alligevel vil man , at Digteren skal fødes med <b>Kundskab</b> til det Uudgranskeligste af Alt , til Menneskehjertet og Naturen . Man troer , at Digterevnen selv er Begeistring , og drømmer om en lykkelig Tilstand , hvori Poeten efter et Slags Dictat fra Himmelen halv übevidst skal nedskrive Ting , hvorom han selv ingen Anelse har , og hvori han røber Erfaringer, som ban ikke har gjort . Dette falske Begeistringssnak , der fra Begyndelsen kun er en affecteret Udtryksmaade fra det forrige Aarkundredes Digtere , er for længe siden gaaet af Brug ( uden i Morgenbladet ) , og jeg har aldrig hørt , at nogen Digter har troet , den var sand . Det er en simpel Misforstaaelse ; det er ikke saa let at være Digter . Den dybere Evne til at modtage poetiske Indtryk , til at gjøre disse Erfaringer , til at erb verve

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008090903035'target=_>Andresen, K.N., 1848, Haandbog i Afholdsreformens Grundsætninger</a><h4>334</h4>en stor maasiee den storste Deel iblandt dem hcnlevcr sit Liv , og saa at sige kveler enhver Spire til Sedelighed , Velstand og Orden , som Kirke og Stat nedlcegge med saamegct Haab . Disse ere de Brendeviinsdrikkcns uudeblivelige Folgcr ( der ri kunne forebygges , saalcenge Vrendeviinsdrik er til ) , som med Grund opfordre til sercgne Foranstaltninger mod Brendcvinets Brug som Drik , ikke Brendevinets berusende Kraft i og for sig selv bctragtct eller den Omstendighed , at man ved at bruge det som Drik af og til- kunde faae sig et Nuus . Grunden , den ene tilstrekkelige Grund til her at skride ind og ved sceregne Foranstaltninger at optrede mod Vrccndeviinsdrikken ligger altsaa ikke i Vrendevinets berusende Kraft , men i den eiendommelige Maade , hvorpaa Brendeviinsdrikken, sammenlignet med andre berusende Drikkes Brug , foregaaer . Denne er det nemlig , der , som ovenfor bemerket , gjor det begribeligt , at et Forbrng af 20 Millioner Potter i et Land med Norges Folkemengde kan finde Sted , og det er formeentlig aldeles ufornodent for den med Forholdene bekjendte Leser at gaae ind i Dctaillen med Hensyn til den Maade , hvorpaa Brcendevinet bruges fornemmelig af de saakaldte simplere Klasser og navnlig i Vycrne . Det er ei et Nuus ved enkelt Leilighed, hvorom her er Tale , men et Nuus idetmindste et til det Redsomme stigende Forbrug hver Dag i Aaret , den ene Dag som den anden , fra Morgen til Aften , et saa at sige st a a ende indbildt Styrkelsesmiddel , der antagcs at hore med ved alle Lcilighcder i det daglige Liv , midt under Hverdagslivets Beskjeftigelser , ja endog forst og fremst ved disse , en Drik , der til enhver Tid med Begjerlighed sluges af Oldinger , Mcrnd , <b>Kvindcr</b> og Born , der ligesom danner Middclpunktet for de simplere Klassers Dnsker , Planer og Forhaabninger . Det er dernest en aldeles urigtig Forestilling at tenke sig Vrendeviinsoverdaadigheden hos de simplcrc Klasser som bcstegtet med og ligesom en mod lavere <b>Formaal</b> rettet Mtring af dm blandt de ovrige Stender ikke sjelden stedfindcnde Tilboiclighcd til Lurus . Den store Forskjel er nemlig ven , at der her nesten altio iretmindste til en <b>vis</b> Grad sinder en

<h4>744</h4>1. Den forste og vigtigste Betingelse for en Afholdsforenings hcnsigtsmcessigc Virksomhed er den , at Foreningen selv klart opfatier sit Vcesen som en Foranstaltning ei mod Fyldcrict men mod selve Broendevinets Brug som Drik i ringeste <b>Maal</b> , altsaa ei mod den umaadelige men mod den maadeholdne Nydelse . At bringe den maadeholdne Brug , af Brcendcviin og deraf tillavede Drikke til lidt efter lidt at ophore , er Afholdsforcningernes eneste <b>Formaal</b> . Dette kan igjen alene bcvirkes derved , at efterhaanden Flere og Flere nctop iblandt de allerede iforveien Afholdne eller idetmindste Wdruelige indtrcede i Foreningen; chi Indtrcedelsens Hensigt er ei at silkre sig selv mod en bcfrygtet Fare i denne Henseende , men alene den , offentlig at udtale sin Mening i denne Sag , d . e . offentlig at lccgge sin Misbilligelse med Brcrndevincts Brug som Drik navnlig hos den opvorcnde Slcegt for Dagen . Som en Folge heraf maa enhver Forening i kraftigste Maadc protestere mod den endnn fast ligesaa almindcligc som grundfalske Mening , at det fornemmelig er drikfceldige Personer , hvis Indtrcedelse onskcs . Da Erfaring viser , at hverken Skrift eller Tale formaae at adsprede den Taage , der endnu hviler over den hele Reform i denne Henseende , er der intet andet Middel tilbage for Forcningcrne end i Gjerningen at <b>vise</b> , at Tingen ikke er at forstaae paa denne Maade . Man afvise derfor stcdse og ved enhver Leilighed ( gjenncm Foredrag i Foreningens offentlige <b>Moder</b> saavelsom ved private Anledninger , hvor Nodvendigheden byder det ) det gamle grundlose Snak om  det Dnstcligc i , at Den eller Den

<h4>764</h4>Salg , Brcending og Trakteren hos Andre o . s . v. Man stoder saaledes ikke sjelden paa Mcend , som ganske og aldeles billige dm hele Foranstaltning og gjerne vilde bidrage , hvad de kunde , til Qiemedets Fremme . Med alt dette seer man alligevel , at de ikkedestomindre ikke indtrcede . Spsrger man dem om Aarsagen hertil , faaer man altid det samme Svar :  leg skulde gjerne indtrcede, da jeg alligevel ikke drikker Brcendeviin , dersom jeg kunde virke noget for Sagen . " Dette , , Virkc " er et ganske bcfynderligt Ord i Afholdsanliggendet ; man horer det meget hyppig ved forskjellige Anledninger , hvor dette Anliggende omtales , og dog er det vist , at det i den Betydning , hvori man ved siige Lciligheder gjerne forstaaer det , allermindst hsrer hjemme i dette Anliggende ; thi Sagen staaer virkelig saaledes : der er Intet at gjore , at sige forMcengden af Foreningerncs Medlemmer . Det saagodtsom Eneste , der er at foretage , bestaaer i ved Skrift og Tale at bidrage til at fremme vedkommende Forenings <b>Formaal</b> , hvoraf man er Medlem ; det Forste kan ikke vel tcenkes at falde uden i Enkeltes Lod , da i modsat Fald Verden vilde blive altfor opfyldt med Afholdsskriftcr ( hvilket den i en <b>vis</b> Henseende maaskee allerede er ) ; det Sidste i Forcningerncs offentlige <b>Moder</b> , at holde Taler eller Foredrag , er naturligviis Noget , man ikke kan forlange af Enhver iscer , da man i modsat Fald ogsaa i denne Henseende fik Formcget af det Gode . Den megen Tale om  at virke for Sagen , " forsaavidt det ikke bestaaer i een af eller begge de nccvnte Virkemaader , har ncermest sin Grund i den samme urigtigc Forestilling , der fremkalder saagodtsom alle Misforstaaclser i denne Sag , nemlig at det her gaacr los paa at bidrage til Vrcendeviinsdrikkens Formindskelse eller Indskrcekning . Men det er netop denne man som Medlem af en Afholdsforening slet ikke har med at bestille . Afholdsforeningerne gaae ei ud paa at indstrcenke Vrcendeviinsdrikken ( d . e . paa at bringe de Brcendeviinsdrikkende til at drikke mindre end for ) , men paa at afskaffe den , d . e . at bringe det dertil , at stedse Flere og Flere beslutte sig til aldeles ikke at drikke Brcendeviin og , hvad der for Foreningerne er det Vigtigste , ved sin

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009071303007'target=_>Mack, F., 1865, Alfred Fjeld</a><h4>1113</h4>kastede over hendes Tilverelses Vane , og som havde givet Livet sit storste Verd for hende , var udslukt og aflost af dm morke Nat . Dm Kjerlighed , som hun troede , at Fjeld neredc for hende , havde gjort hende saa lykkelig ; hun havde saa tillidsfuldt overgivct sig til Troen paa den , og denne Tro havde gjort Livets Byrder lette , havde anbnet ru ny og herlig Verden for hende , hnvde brngl hende til nt se alt i rosmfarvet Skjer og havde fyldt hende med Haab om en ren og üblandet Lykke , og nu saa hun , at del kun var en Illusion , at hun havde bedraget sig selv og var bleveu bedrage » , . Frygtelige Villeder fremstillet » . - sig for hendcS <b>Sjel</b> ; i Vegyndelsen oincdc hun iutet Hnnb , ingen Trost . Fremtiden stod mork og truende for hende og lovede hende et Liv ndeu Glede , uden Interesser og nden andet <b>Maal</b> , mt » Dodm, et frygteligt Villede for en livsglad , ung <b>Pige</b> . Der . for gav hendes kraftige <b>Aand</b> ogsaa i det fsrste Bieblik ester for Smerten ; Fortvivlelsen greb heude med sine Arme , omslyngede hende og knugede hende tettere og tettere sammen ; rn Folelse af det Umulige i at kuuue udholde denne navnlose Smerte paatvaug sig heude og fyldte heude med Nedscl , og i Vaggruuden viste Afsindighcden sig i et mydrligt Taageslor som et strekkelig ! Spogelse , der uermede sig mere og mere og truede med at gribe hende .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112000022'target=_>Stenersen, Stener Johannes, 1863, Professor Stenersens Reformationshstorie</a><h4>295</h4>gammel , til Magdebmg og Aaret ester til den endnu mere blomstrende Skole i Eisenach . Iblandt de Lcrrere , hvoraf Luther her blev undervist , ncrvnes iscrr en Johannes Trebonius, der lcerte Drengene Grammatik , og hvem Luther siden roste saavel for hans Forstand , Lceremethode og <b>Lcerdom</b> , som og for den Agtelse , hvormed han behandlede Drengene ; thi han pleiede hver Morgen , naar han kom paa Skolen at hilse dem med blottet Hoved , og at straffe sine Medlcrrere , naar de ikke gjorde det ;  mellem disse Drenge " , sagde han , sidder Folk , af hvilke Gud gjor Borgemestere , Cantslere , Doctorer og Dvrighedspersoner, omendftjsnt I ikke nu se det ; disse bor I billigen <b>vise</b> Wre . " Saavel her , som i Magdeburg maatte den unge Luther erhveive sig Livets Underhold paa den kummerlige Maade ved at synge udenfor Borgernes Dore , og ofte kom den arme Dreng derved i en saadan Nod , at han tcenkte paa at drage hjem igjen , ligesom ogsaa denne Maade at erncrre sig paa var nedsiaaende for den crdeltsindede Drengs Gemyt . Men denne hans Nsd endtes , da Conrad Cottas <b>Kone</b> , rsrt over Drengens tunge Kaar og over den Inderlighed , hvormed han bad , tog ham i sit Hus og , som man siger , til sit Bord . Alligevel kom den unge Luther , skjent han i Skoletiden havde fristet saadan Trang , med gode Forkundstaber til Hsistolen i Erfurt iFOI , hvor han fornemmelig fik en lodocus Isenacensis til Lcerer , som var en saa brav Mand , at Luther stedse for ham bar stor Heiagtelse , og som tillige var en Mand , der ikke var blottet for al Indsigt i den sande Theologie ; i det mindste siger Luther i det deilige Brev , som han strev til denne Lcerer , da han laa syg , at det var ham , som havde lcrrt ham , at man allene stulde tro de canoniste Boger ligefrem , men alle de andre kun ester Prsvelse . Ogsaa fik han i lohann Grevenstein og Gerhard Hecker ikke almindelige Lcerere . Under disse Lcereres

<h4>1332</h4>tro og haab paa Gud Fader den almcegtige , den rette Ertsstaffer , i hans Sons Navn , der ved sit Ord braqte Solv og Guld i Mens Mund , som endnu stedse lader Ertssen voxe og giver den i rette Tid til dem , som holde ved i deres Gruver , og holde med hjertelig Bon ud hos ham i Taalmodighed . Den evige Gud vil verre med Eder og paa hans rige Velsignelse og milde Haand afioser jeg Eder ogsaa fra al Eders Skyld . " Men forend denne Mand kom hjem , kom der Vudstab til ham paa Veien , at man havde truffet paa god Erts i hans Grube ; Arbeidet gav Udbytte og Manden lsste sig virkelig ud af sin Gjeld . Ved denne Leilighet » fortjener og den Tildragelse at omtales , som Katholikerne sagde var Aarsag i hans hastige Dod . Man forte nemlig en <b>Pige</b> paa 18 Aar til Luther og bad , at han vilde uddrive Djcrvelen af hende ; thi de sagde , at hun var besat . Luther bad hende lase Troesartiklerne , det gjorde hun ogsaa , men da hun kom til den anden , taug hun stille og blev gruelig plaget af Djcrvelen . Luther sagde da :  leg kjender Dig vel . Du Djawel ! Du saa gjerne , at jeg med store Ceremonier vilde besvcrrge Dig , men jeg vil ikke gjsre det . " Da Luther Dagen ester prcedikede , blev hun bragt i Kirken og efter Prcrdikenen md i Sakristiet , hvor hun faldt til Jorden og siog med Hcrnder og Fsdder . Nogle Studenter reiste hende op igjen , hvorpaa Luther erindrede de Tilstedeværende om , at den Magt at uddrive Djcrvle ei lcrnger var i Kirken , at den ei heller nu , da Kirken allerede var plantet , lcenger var fornoden , at man ei heller ffulde bryde fig om de pavelige Ceremonier , men bede mod denne <b>Aand</b> og ikke scette Gud <b>Maal</b> eller Grcrndse , naar han flulde uddrive den , da Gud derved blev fristet : meget mere skulde man fare fort i Bonnen og med Taalmodighed vente den af Gud bestemte Hjcrlpetime . Han lagde derpaa Haanden Paa <b>Pigens</b> Hoved , ftemsagde Fadervor og Troesartiklerne

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010021803021'target=_>Smith, C. W., 1858, Om Holbergs Levnet og populære Skrifter</a><h4>506</h4>ikke tilstrekkelig kunde bestytte . Hertil hprer fornemmelig Rangsygen . Dette Raserie er Gjenstand for Satiren i Comedien  dcn honnete Ambition " . leronimus , Stykkets Helt , er som scedvanlig en gammel Borger i Kjpbenhavn . En yngre Mand af hans Slcegtstab , har ved Connexioner skaffet sig en Rang , og Tanken om , at denne nu rangerer over ham , foruroliger ham i den Grad , at han hverken kan spise , drikke eller sove . For at faae Ro i sit Sind maa han have en Rang ved hvilkesomhelst Midler , naar de blot spre til <b>Maalet</b> . Denne sin Lidenskab kalder han med et formodentlig blaudt Rangsyge dengang brugeligt Udtryk en honnet Ambition . Det nytter ikke , at hans ligesaa fornuftige som fromme og fpielige <b>Kone</b> foreholder ham , at hans yngre Slcegtning ved sin urimelige Rangsyge kun har paadraget sig fornuftige FoM Foragt , og at det vil gaae ham selv ligeledes . Saadanne fornnftige Folk ere i hans Mne Svidsborgere , der ingen honnet Ambition have ; det er ham knn om at gjpre , at de blive tvungne til at <b>vise</b> ham ndvortes 3 Erbodighed , hvormcget de end maatte foragte ham i deres Hjerter . Dog ønsker han ogsaa , saavidt muligt , at undgaae det Sidste , og har derfor i Sinde at skyde Skylden for sin Rangsyge vaa sin <b>Kone</b> , idet han vil sige til sine Bekjendtere , at han meget imod sit Vnske seer sig nodt til at spge om en Rang for at fpie sin <b>Kone</b> , som absolut vil vcere Frne . Han undseer sig ikke for at bede sin <b>Kone</b> om hendes Minde hertil , idet han forestiller hende , at Sligt ikke lcegges <b>Fruentimmerne</b> saamcget til Last som Mandfolkene , og hun er virkelig fpielig og godmodig nok til at paatage sig Skylden for sin Mands Svaghed . For at skaffe sig Anbefalinger forspger han de besynderligste Midler . I det Haab at kunne komme i Forbindelse med en hpitstaaende Mand byder han dennes Lakei til Bords , om hvem han har hprt , at han stal have stor Indflydelse hos

<h4>632</h4>erindrer om to grceske Ord , hvorved det kommer til at betyde : den , som ringeagter Verden . Dette er nemlig den Charakteer , han indbilder sig at bcsidde . Han har studeret Theologien og dm dermed forbundne Philosofthie og mere end scedvanligt var blandt Datidens Studerende , der i Neglen kun studerede for Brpdet , fordybet sig i disse Videnskaber , navnligen i den sidste ; han har derfor ogsaa taget Doctorgraden med stor akademisk Berammelse . Som det ofte er Tilfceldet med dem , der eensidigt fordybe sig i philosophiske Studier , stikker han ikke dybt i egentlig <b>Lcerdom</b> ; Grcesk forstaaer han ikke , og Latinen trykker ham ikke meget . Men det er heller ikke dette Slags Dygtighed , hvori han scetter sin 3 Ere ; han vil ikke vcere lcerd , han vil vcere viis . Den Foragt for Verden og Fordybelse i sig selv , som deels den christelige Pietisme , deels den gamle stoiste Philosophie hver paa sin Maade anbefaler , har saaledes indtaget hans Gemyt , at han har gjort den til en Opgave for sit Liv . Han er ingen Hykler ; han troer sig virkelig sikkret mod Fristelser og fri for Lidenskaber , fordi han ingen Lyst har til Baller , Maskerader , Comedier , eller andre saadanne verdslige Daarskaber , som han idelig ivrer imod ; for at <b>vise</b> sin Uafhcengighed af Verdens forfengelige <b>Moder</b> bcerer han en afstikkende gammeldags Dragt , trekker sig tilbage fra selskabelig Omgang og skyer og laster alle Fornpielser som ligemeget stridende mod Religion og Philosophie . Han lever for sine Studier og for en Selvbetragtning , hvis Resultat er meget tilfredsstillende , da han er fast overbeviist om , at ingen Affect kan virke noget hos ham , og at Vrede , lalousie , Begjerlighed efter Penge , Begjerlighed efter LEre for ham ere afmcegtige Fjender . Desmere smerter det ham at see sine Medmennesker beherskede af alle disse Lidenskaber , og af disse halv andegtige halv philosophiae Betragtningcr kan han blive saa betagen , at han

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2007102902001'target=_>Steffens, Henrich, 1841, Hvad jeg oplevede</a><h4>246</h4>Hope , i Samnanger-Fjord , for at beftge en anfeet Mand , til hvnn jeg var anbefalet . Han heed Christie , havde beklcedt betydelige Mmbeder og nu , som en formuende Mand , trukket sig tilbage i Stilhed. En anseelig Skikkelse , med en Holding , der bod Agtelse , og vidnede om et. tidligere Liv i store Kredse , traad mig imode og bod mig medVenlighcd velkommen . Saavidt jeg kan mindes , levcde han alene , idet mindste mindes jeg ingen andre Bebocre , iscer ingen <b>Damer</b> . Huset var elegant indrettet ; han behandlede mig venligt og forekommende , dog med en <b>vis</b> Vcerdighed , paa en faderlig Maade , der bod mig Befledenhed og Maadehold i mine Ittringer . Hans vidcnfiabelige <b>Dannelse</b> glcedede , hans modne Domme imponerede mig . Det var mig velgjorende , at sinde en Mand her , som jeg i enhver Henseendc maatte scrtte over mig selv , og den Orden , Reenlighed, Beqvemmelighed , de fortrinlige Maaltider og de fortreffelige Vine havde jeg tilstrcrkkeligt lcrrt at siatte , cftcr at jeg allerede i nogle Dage havde drevet omkring i de smudsige Fisterhptter . Han vilde <b>vise</b> mig en Mon Udsigt , og forde mig ind i en Sal , hvis Elegants ovcrraflede mig . Denne var i sin Lamge bcelt i tre Dele ved to Ssilerader , og hvad

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016030829005'target=_>Gislesen, Henriette, 1851, Helligaftenerne</a><h4>107</h4>vagelse , da veed jeg det saa vist , saa vist , at hendes barnlige Leg var Herren dyrebar , at ogsaa den var en Gjerning , hvortil Han gjerne havde hengivet Brngen af sit hellige Navn . Saaledes som hun legede maa Han Selv , da Han var Barn have leget , kun endnu frommere og elskeligere . Ruth . Men Han var jo Guds Son og Selv Gud , og har været med at stabe alle , alle Ting / som <b>Moder</b> ' fortalte os i Julen , en rigtig stor , almcrgtig , Gud . Det var da ikke derfor vist , Marie , at Han skulde have leget som Barn , fordi Soster Stefanie gjorde det . Moderen . Nei , ikke derfor ; ikkeheller som Stefanie eller som noget andet Barn har Han leget . Det kunne vi vel slutte os til af hine Ord i vort forrige Sondagscvangelium, der udtales om Vorherre , allerede som lidet Barn : " Men Barnet vorte og blev stcerkt i Aanden , fuldt af <b>Visdom</b> og Guds Naade var over Ham . " <b>Aand</b> , <b>Visdom</b> og Naade har hvilet over Ham fra Hans spædeste Barndom , Alt vel kun i noicste Overensstemmelse med hvad Hans Alder , eftersom Sjæl og Legeme udviklede sig , tiltrcengte ; men dog stedse i en saadan Fylde , at den , igjennemtrcengende Hans hele Vcesen , ogsaa har udtalt sig i enhver Livets Ilring , selv den miudste og übetydeligste og derved gjort Hans Fremtrceden forskjellig fra ethvert andet Barns . Dersom vor Herre Jesus levede som Barn hernede nu paa Jorden og det forundtes et Menneske , der førstod at tyde Aandens og Naadens Prceg ogsaa i Barnets Udtryk og Vcesen og hele Maade at være paa , — dersom det forundtes en Saadan, at se Ham i Leg med andre Born , kunne I da tvivle et Oieblik om at dette Menneske , uden et Sekunds Bcta-nkning , vilde vide at udpege for Eoer det guddom-

<h4>758</h4>Lcegger kun Mcerke til hvad Han gjorde , da Han blev efterladt alene i Jerusalem , da de Mennesker , hvilke Gud havde overdraget at vogte Ham til Sjæl og Legeme vare borte og Han var overladt til sig Selv . Han overlod sig ikke til Angest eller Mismod , lob ikke cengstelig hid og did iblandt Menneskene for at soge Trost og Hjcrlp , endnu mindre benyttede Han den Omstændighed, at Hans jordiske Paapassere vare borte , til nogensomhclst utilladelig Benyttelse af Friheden , til nogen selvraadig Nydelse af den . Han overgav sig , strar da Han fornam at Han var alene og ikke lamgere var vogtet af menneskelige Oine , med Sjæl og Legeme til Gud , til sin himmelske Fader , for at Denne kunde " give Naad over Ham med sit Oie , " for at Denne kunde lede og stjærme Ham og <b>vise</b> Ham Veien , hvorpaa Han skulde gaa . O , hvilken dyb <b>Lcerdom</b> ligger der dog ikke for det ungdommelige , til sig selv overladte og enestaaende Menneskehjerte i denne Beretning om Herren! Er det ikke som om den var optegnet til hellig Formaning og Trost for Alle , af Fader og <b>Moder</b> for-

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2007060412005'target=_>Rosenius, C.O., 1879, De ti Guds Bud</a><h4>339</h4>Men fornden det mrvnte store Hovedsiemed med Sabbaten, vor Beredelse for det evige Liv , saa have vi endnu at takke Herren for nogle scrrskilte Velgjerninger i denne Indstiftelse . Da Gnd ikke blot i almindelige Ord anbefalede os denne saare magtpaaliggende Sag , men ogsaa bestemte et vist <b>Maal</b> af Tid , som skulde auv endes til vord evige Vel , saa har han ikke blot i Almindelighed kommet os til Hjelp imod vor faldne , jordisk sindede Natur , men har swrlig tcrnkt paa dem iblandt os , som staa under andres Myndighed , saasom Bsrn , Tjenere og Underordnede , og vaaget over deres Ret til <b>Sjelens</b> Erncering med Guds Ord , idet Herren i det tredie Bnd forbyder enhver Husfader eller <b>Husmoder</b> paa Hviledagen at forhindre sine Tjenere fra Guds Ord med Ugens fcrdvanlige Arbeider . Herren siger :  Sex Dage skal du arbeide og gjsre alt dit Arbeide . Men den syvende Dag er Sabbat for Herren din Gud ; da stal du ingen Gjerning gjsre , hverken

<h4>416</h4>ogfaa at hedre dem — hvilket foruden Kjerligheden indbefatter i sig ogsaa adskilligt endnn Hsiere , saasom Mrbsdighed, Idmyghed og Lydighed . Saadaut tilkommer ikke Foreldrene for nogle deres tilfeldige Fortjenesters og udmerkede Egeuskabers Skyld , ikke heller for de store Velgjerningers Skyld , som vi have nydt og nyde as dem i rigere <b>Maal</b> end as andre Mennesker . Nei , den Heder , Gud har befalet os at give Foreldreue , tilkommer dem blot for den Egenskab , at de ere Foreldre , at den Herre Gud har sat dem over os , for at trede i hans Sted og indtage hans Plads herneden , og befalet os at hedre dem . Det er derfor hans kjerlighedsfnlde og <b>vise</b> Villie , som ligger til Grnnd for Foreldre-Hederens Retmessighed og Nsdvendighed . Derfor bevidner Gud ogsaa sit store Velbehag i Lydighed mod Foreldre , at den er  velbehagelig for ham " ( Col . 3 , 20 ) , og fsier til det fjerde Bnds Efterlevelfe en serskilt Forjettelse om Velferd og langt Liv paa Jorden . Vi have fslgelig ikke at indlade os paa nogen Prsvelse af Foreldrenes Beskaffeuhed og Egenskaber , for at nnderssge , om de fortjene vor Heder eller ikke , men vi have blot at efterspsrge , om de ere Foreldre , om de staa til os i Foreldreforhold , om Gud har sat dem over os ; thi da ere vi strax skyldige til at bevise dem Heder , hvordanne de end for fine Personer eller i sit Forhold til os kunne vere . Et Barn kan derfor ikke vere vissere paa , at det gjor Guds Villie , end naar det hedrer , elsker , agter og lyder Fader og <b>Moder</b> ; thi dette har den Herre Gud selv befalet og har , bevidnet sit serdeles Velbehag deri . Tenk , hvilken stor Naade og Velgjerning ogsaa dette Bud indeslutter i sig , at et Barn , som ellers ikke kan foretage store og lysende Gjerninger i denne Verden , kan paa saa nert Hold og paa en saa let Maade gjsre Gjerninger , om hvilke Gud tydeligere end nogen anden har tilkjendegivet sit Velbehag , nemlig blot ved at vere sine Foreldre underdanig og lydig og have Mrefrygt for dem .

<h4>431</h4>som fremmede for hverandre . Naar nu Foreldrene derhos blive slsve , paa Grund af Alderdom lide Tab paa Sjelskrefter, Hukommelse og Forstand , ja  gaa i Barndommen " , samt under alt dette <b>vise</b> Knarvurrenhed og ondt Lnne , saa at Intet af hvad Barnet gjsr er dem ret tilpas , og de alligevel uindskrenket ville sve sin Forcrldre-Magt over Bsrnene, da gjelder det at fornegte sin egen <b>Visdom</b> , ja , i Alt at fornegte sig selv , hvis man nu skal kunne  ere Fader og <b>Moder</b> med Taalmodighed " . Bor ikke da Guds <b>Aand</b> i Barnets Hjerte , saa at det drives as denne hellige Frygtens og Kjerlighedens <b>Aand</b> , da gaar det som man saa oste ser , Barnet trettes ved Foreldrenes knarvurne og besværlige Vesen og begynder ikke blot at foragte dem i sit Hjerte , men ogsaa at lade denne Foragt aabenbare sig i Fagter , Ord og Gjerninger , saa Foreldrene dybt saares og fortsrnes derved . Ja , endog hvor Guds <b>Aand</b> bor i Hjertet , men man enten ikke agter paa hans venlige Advarsler, eller man paa Grund af forudgaaende feilagtig Opdragelse er bleven slet vant , kan der i denne Retning syndes paa den forferdeligste Maade .

<h4>452</h4>I Sandhed , uden dem skulde Verden blive helt og holdent en Rsverknle . Men saa meget nnaturligere er den Synd at nnddrage sig fra Foreldrenes Tjeneste , naar de behove den . Hvor mangen Ssn og Datter er alligevel ikke ugudelig nok til at anse det for en Byrde at behsve at <b>vise</b> Fader eller <b>Moder</b> nogen virkelig Tjeneste ! For sine Bsrn kan man mnligens gjsre Alt , men for sine Forcrldre vil man Intet gjore . Man snsker dem kanske heller Dsden og gleder sig , naar de ere borte . Eller om man tjener dem , saa gjor man det blot med Ulyst og forbitrer derved Foreldrenes Brsdkrnmmer , maaske bereder dem en for tidlig Grav . Kjerre , vil du , at dine Bsrn i din Alderdom og paa Nsdens Dag skulle gjsre det Samme mod dig ? Vil du det ikke , saa tjen med Villighed og Glede din Fader og <b>Moder</b> ; thi da skal der blive dig maalt med det samme <b>Maal</b> . Avrighed og Lerere knnne vi i Serdeleshed tjene med Forbsnner , hvortil Ordet ogsaa formaner .

<h4>632</h4>hende ( d . e . med Lyst og Forsagt til Synden ) , har allerede bedrevet Hor med hende i sit Hjerte . " I Lyset af denne grundige Forklaring skulle vi bctragte dette Bud . Men lader os fsrst underssge , hvorledes det kan verre at forståa , at nagtet dette Bud har en saa dyb <b>aandelig</b> Betydning, at det er overtraadt blot med en nren Begjcering og omfatter alle den urene Lysts Rorclser og ytringer , saavel i som ndenfor Mgteskabet , ( hvilket mange Skriftsteder bevise ) , det dog egentlig lyder saaledes :  Du skal ikke bryde VEgteskabet . " Der ligger uemlig en dyb <b>Lcrrdom</b> angaaende dette Bud indesluttet heri . Ja , hermed fsres vi tilbage til den fsrste , egentlige Grund for dette Bud , nemlig Guds i Menneskets Skabelse ndtryktc Plan og Tanke med Hensyn til dets Tilværelse og Forplantning paa Jorden, at han nemlig skabte en Mand og en <b>Kvinde</b> , og strar , ja endnu fsrend <b>Kvinden</b> var skabt , besluttede VEgtcskabcts hellige Indstiftelse . Menneskets Skabelsc og Slcrgteus Formerelse vare tuende hos Gud foreuede Tanker ; og derfor finde vi Wgteskabets Stiftelse , som er Grnndlaget for det sjette Bud , i Bibelens forste Capitel , i Skabelseshistorien V . 27. 28 :  Mand og <b>Kvinde</b> skabte han dem . Og Gud velsignede dem , og sagde til dem : vtrrer frugtbare og mangfoldige " — og derfor tilfsies i det andet Capitel :  Derfor fkal Mandcn forlade sin Fader og siu <b>Moder</b> og blive faft hos fin Huftru , og de skulle vcvre eet Kjsd . " sHer se vi ogsaa Gruudeu til den Ordning af Budene , at strax efter det Bud , fom tåger vort Liv i Beskyttelse , folger det Bud , som vcrrner om deu hellige Indstiftelse til Slwgtens Formerelse). Og det er knn ved den tilborlige Mrefrygt for

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010052806001'target=_>, 1867, Det nye Testamente med Forklaringer</a><h4>1750</h4>faa hundrede <b>Maal</b> Jord igjen , naar de for Evangeliets Skyld forlade et <b>Maal</b> Jord , eller at de , naar de give en Daler til de Fattige , just stulle faa hundrede Daler istedet ; var det saaledes ment , faa vilde Vejen til Himmelen vcere bred og blive betraadt af Mange . Nej , indvortes i Hjertet forcgaar ved Omvendelsen et Under , hvorved Alt , hvad man som Kristen modtager , bliver os hundrede Gange , ja vel Millioner Gange mere voerd end det , man befad fsr sin Omvendelse . Og hvad er da dette for et Under ? Man faar Taknemlighedcns <b>Aand</b> . Nu takker man den himmelske Fader for store og fmaa Gaver , fordi man ved , at man ikke har fortjent Noget af det , som Han tildeler En ; man feler og sinder den kjcerlige Kristi Haand i alle Ting . — Og opfyldes ikke ogsaa Forjcettelfen , hvad Ens Nærmeste angaar ? Vi ville anfsre et ' Erempel om de forladte og de paa mangfoldige Maader igjen crholdte Bwdre . I have omvendt eder ; da maatte I opgive eders verdslige Omgang , eller meget mere , fordi I ikke mere kunde fslge med eders Kammerater i det tomme , vilde , uordentlige Liv , have de udswdt eder . Det gjorde eder i Begyndelfen meget ondt , thi I stode jo ganske alene ; men nu have I faaet andre Brsdre og Venner , hvilke I elste med en Kjcerlighed , om hvilken I tidligere ingen Anelfe havde . Ja , er ikke den Kjcerlighed , fom I for lefu Skyld have til Mand , <b>Kone</b> og Born , felv om de endnu ere vantroe , hundrede Gange stcerkere og inderligere end den kjsdelige Kjcerlighed , som I maa forsage ? Se , da er jo Frelserens Forjcettelse opfyldt paa eder ! " B . V . 30. . . Den Herre lefus har ikke forholdt Disiplene Noget af det store Haab , som deres Kald stjenkede dem , men foreholdt dem her ' Skatten i Himmelen ' fra den herligste

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112101089'target=_>Truvé, T., 1876, Moody og Sankey og deres Virksomhed i Skotland, Irland ogEngland</a><h4>169</h4>Naaden . Men alligevel stal Lovens Fordring fuldbringes i os , som ikke vandre efter Kjedet ( thi det er under Loven ) , men efter Aanden ( Rom . 8 , 14 ) . Hans tydelige Skjelnen mellem Lov og Evangelium har vcrret til stor OvlMelse for mangen betrcengt Kristen . Vore amerikanske BrMre have tillige gjort stort Gavn ved at <b>vise</b> os , hvad der kan gjøres for Sjcelenes Omvendelse, naar man med hele sit Hjerte lever sig md i denne Gjerning . De kom til os med dette <b>Maal</b> for Pie , Herren gav dem , hvad deres Hjerter begjcerede , og som F ^ lge heraf ere Sjcele i hundredevis blevne omvendte . Alt andet gave de slip paa . Hr. Moody bekymrede sig kun lidet om de scedvanlige Former i Livet , naar han kun kunde vcekke en bekymret Sjoel , som han frygtede ellers vilde fortabes . Det traf sig undertiden , at hans Venner ved MFdets Slutning gjorde ham bekjendt med en eller anden fremragende Personlighet » : men de bleve ncesten stedte over , at han istedetfor at tale med denne skyndte sig bort til en bekymret Sjcel ; men disses Frelse var hans <b>Maal</b> , og alt andet maatte staa tilbage . Dersom vi , som ere Predikanter, havde denne Tro , og virkede ligesom vore amerikanske Brødre for Sjcelenes Frelse , skulde vor Gjerning visselig lykkes os . De have vcrret Redskaber til i denne By at frelse hundreder af Sjcele , der vare ncerved at fortabes . Blandt disse findes mange unge <b>Kvinder</b> og <b>Piger</b> , der i Fremtiden skulle udfylde en v. igtig Plads som Hustruer og M ^ dre . Og dersom de tusinder af nyomvendte , der vare ncervcerende og hMe Moodys Afstedstale , blive bestandige , vil det vcere en uudsigelig stor Velsignelse , Gud har stjcenket vort Folk ved disse Mcend .

<h4>534</h4>oftfyldte Londons kristne , da de vidste , at Moody og Sankey vilde komme til Verdens største Stad . I en lang Rcekke Vogne , trukne af det hurtige Lokomotiv fra Liverpool til London , findes Mcend og <b>Kvinder</b> af alle Stcender og Klasser , med forskjellige <b>Maal</b> for sin Reise . Nogle komme forat s ^ ge Rigdom , andre forat s ^ ge Fornyelse , og deres <b>Formaal</b> ere , ligesom de selv , mere eller mindre gode . I en Vogn sidder to Mcend , hvis ydre ikke stiller dem fra de Mige reisende . For mindre end to Aar siden bef ^ gte de Staden York og kjendte ikke et Menneske . De holdt nogle gudelige M ^ der der , ved hvilke der kun vare nogle faa tilstede , og hvorledes de endda kunde famle disse , vide vi ikke . Hvilken Text ) de valgte , vide vi hellerikke . Da de vare simple Moend , var det ganske naturligt , om de maaske Iceste ftrste Del af Pauli ftrste Brev til de Korinthier :  Thi betragter , Brødre ! eders Kald , at ikke mange <b>vise</b> efter KjKet , ikke mange mcegtige , ikke mange fornemme ( ere kaldede ) , men hvad der er daarligt for Verden , har Gud udvalgt , forat bestjcemme de <b>vise</b> , og Gud haver udvalgt , hvad der er strFbeligt for Verden , forat bestjcemme det stcerke " . De have reist en lang Vei forat holde disse Mcher . Hvorledes stulle de kunne overvinde , hvad der er mcegtigt , cedelt og vist i Verdens Pine ? De VFie Knee og bede i Troen . Nogle Maaneder efter se vi dem ncerme sig London . De ere bekjendte for tusinder af Mennesker , femhundrede Prester af alle Samfund have holdt M ^ der forat berede sig og Folket til deres Ankomst . De stMste Huse og Sale ere leiede eller for Tilfceldet byggede forat rumme de Masser af Mennesker , som man venter ville stramme til for at h ^ re . Disse Mcend komme nu med sine Vaaben forat indtage en Stad med fire Millioner Indbyggere . Den ene har en Bibel , den anden en Sangbog . Den ene taler og den anden synger , medens de begge i Stilhed bede til Gud , og den Gud , de tjene , stal bevise , at Seieren vindes ikke ved Krigshcer eller Magt , men ved hans <b>Aand</b> . Samme Dag , de kom , saa man i Verdens største Blade baade i England og Amerika ( til sidstncevnte Land pr. Telegram ) den Bekjendtgj ^ relse :  Idag begynde Moody og Sankey sin Virksomheo i London . Hele Byen synes

<h4>567</h4>dem vcere velkomne igjen nceste Aften og gjorde dem bekjendte wed en eller anden alvorlig og forstandig kristen . Paa Veien fra disse Mcher kunde man hMe Sang paa Sang opstige fra Omnibusser og Svorvogne . Det hcendte engang , maaske flere Gange , fom vi ikke vide , at da nogle kristne kom ind i en Omnibus , samledes der flere og flere af dem , der kom fra M ^ det , saa at der blev fuldt baade indeni og udenpaa . Man istemte en af Sankeys Sange , det viste sig da , at baade Kudsk og Konduktør vare troende , og disse istemte ogsaa Sangen , saa at Vognen syntes at vcere en syngende Omnibus , da den rullede gjennem Londons folkerige Gader . O , gid vi i vort Land kunde faa en Kristendom , der aabenbarer sig idetmindste i ligesaa stor Frimodighed , som Svcergeren viser , naar han haaner Gud ! Om Lørdagen hvilede Moody og Sankey , og andre fremragende Prcester ledede Mcherne . Men Spndagen var for dem en sand Arbeidsdag . Tidlig om Morgenen saa man Skarer af Mennesker skynde sig til den store Landbrugssal forat vcere med i M ^ det for arbeidende kristne Kl . 8. Vi gjengiver her et kort Uddrag af den Tale , Moody holdt til dem . Han tog sin Text i Daniels 12 , 3 :  Men de , som underviste , skulde skinne som den udstrakte Befcestnings Skin , og de , som have retfcerdiggjort mange , som Stjernerne evindelig og altid " . Moody sagde , at enhver vilde gjerne skinne . Gaar du til en <b>Moder</b> , hvis SM er den ftrste i sin Klasse , kan du vcere <b>vis</b> paa , du faar det at h ^ re , fM du gaar : naar en Fader eller <b>Moder</b> har en SM , der hcederlig har gjennemgaaet Universitetet , hvor stolte ere de da ikke derved ! Vi holde alle af at skinne ; dersom vi ere ovrigtige , maa vi tilftaa det . Et Menneske giver mange tusinde Kroner til en By eller til et bestemt <b>Formaal</b> . Af Taknemmelighed forandrer man Byens Navn og opkalder den efter Giveren forat forevige hans Navn ; men hvilken Feiltagelse ! 50 Aar efter ved man ikke , om Byen har sit Navn efter et Menneske eller efter et Bjerg . Giverens Navn er glemt , men hin <b>Kvinde</b> , fom tMte Alabastkrukken over Frelseren , skal erindres til evig Tid . Hvad vi gjere for

<h4>665</h4>er endogsaa blevet bemcerket , at Evangelisterne afvige noget fra Skriftens Lcere om Omvendelse . Jeg haaber dog , at dersom dette er sandt , vilde en venlig Paamindelse formåa dem til at undgaa disse Hindringer for deres Fremgang . " Erkebiskoppen slutter med at anføre Lut . 9 , 50 ; samt bemcerker :  leg glceder mig over , at Kristus proedikes , hvadenten det ster regelbundet eller ikke , og haaber , at Prcesterne endnu dybere ville indskjcerfte de Sandheder , som ere blevne ftemstillede af begge de vcekkende Mcend . " Rev . Harry Jones , Sogneprcest ved St . Georges Kirke , Mre London , udtaler sig i et Brev paa følgende Maade om Hr. Moody og Sankey :  Den Fremgang , de have havt , er lcererig , ikke alene for enhver kristen , men for ethvert tcenkende Menneske . Den har afslMet meget , som hver Prcest iscerdeleshed burde lcegge sig paa Hjerte og drage <b>Lcerdom</b> af . Kristendommen i dette Ords fulde Betydning , synes at trcenge sig md paa enhver . De Skarer , som HM paa Moody og Santey , <b>vise</b> , at der er en Magt , der virker efter en stor Maalestok . Ingen , som har vcrret tilstede ved et af Moderne i Operahuset, kan nogensinde glemme den Følelse , der grev ham , da 5000 Mennesker reiste sig forat synge Lovprisningen og den 100 de Salme . Naar man ser en saadan Moengde Mcend og <b>Kvinder</b> , sammenpakkede i et Rum og lyttende til en Wst , saa kan man , dersom Hjertet ikke er dM for enhver Livsindflydelse , umuligt gaa bort og dadle et saadant M ^ de. Dets Virkninger strcekker sig lcengere end til dem , der selv deltage i det . Det lukker Spotterens Mund at vcere Vidne til Kristendommens Magt , at se 20,000 Mennesker komme sammen paa engang forat bede . Enhver , som vil tcenke , maa kunne indse , at store aandelige Krcefter rFre sig , men mange ville ikke se eller modtage noget Vidnesbyrd derom . Vi se i denne Bevoegelse et levende og betydningsfuldt Vidnesbyrd for den bedste Bog , Bibelen . Hr. Moody synes med hver Dag at skatte den hFiere . Han synes ikke alene strengt at tro paa dens fuldkomne Infpiration, men ogsaa paa Autoriteten af hvert Ord i den engelske Oversettelse . Jeg har draget Nytte og haaber at drage Nytte af hvert Ord , jeg har HM af hans Muntv

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111401049'target=_>Arentzen, Kristian, 1879, Adam Oehlenschlæger</a><h4>2328</h4>vi troe paa den sandeste , kraftigste og veldædigste Maade i egenligste Forstand at dyrke Hr. Oehlenschlågers Genius og sande Hæder ved med djærve Hug , der gjøre os selv ondt i Armen , at kappe 66 vilde Skud og raadne Grene , der ikke blot vansire , men saftudsuge sammes frodige Træ , hvilket , kritisk beskaaret og befriet især fra de mangfoldige fremmede Indpodninger , pleiet med flittigt Sprog- og Kunst-Studium og gjødet med nogle flere nærende <b>Kundskaber</b> , endnu maaske turde vorde det herligste i Idunas nordiske Have . Skulde denne lidt forkjælede Vi-vaß miskjende vor Gartner-Omhu , saa er vor Trøst , at Iduna selv ikke vil miskjende den , thi hun veed — idetmindste vil hun engang vide det — , at aldrig hyldede en samtidig Genius ærligere en anden end vor Forfatteren af de „ Nordiske Di ^ w " . Vel kunne vi ikke tale om en <b>vis</b> Tilbedelse , som vi maa overlade de blindt Forelskede ; men sikkerlig kan ingen ældre Søster oprigtigere elske den yngre , vi turde sige : selv ingen <b>Moder</b> sin Datter , end VaF ^ 6 B6NB Muse 66 N 06 nl6N8c ; n1aF6I-Bl ^ 6. X6to ^ ) fordi denne livfulde, vevre , blomstrende NnB6B Natni-li ^ - Bl ^ j ^ NN6 Gestalt, smukke Øine og blussende Kinder ligesaa limt skattes af vort Blik som hendes indre Godmodighed af vort Hjerte , beklage vi in6til ( ^ 931 N1N6186 ) der ( i Betragtning af hendes Egensindighed i denne Punkt ) gaaer over til Vrede , at hun saa aldeles forsømmer Bin Paaklædning , for ikke at sige : Pynt , og Renhed idetmindste , for ikke at sige : Pænhed . Vi ønskede hende velopdraget i den Grad , hvori hun er lykkelig begavet , ikke mindre tækkelig og anstændig klædt end velskabt , og ligesaa ren og pæn som rød og

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008091003004'target=_>Thiele, Heinrich, 1844, Kirke-Historie for alle Stænder</a><h4>1840</h4>Georg For var en andcegtig streng Mand , der allerede tidlig ansaa sig kaldet as Gud til at prcedike Omvendelse for sine Landsmcend , ligesom fordum de gamle Propheter for Israels Folk . Han vilde samle den lille Hob af Retfcerdige fra denne onde Verden til et Selstad af Venner . Lcerd var han ikke , han foragtede meget mere al <b>Lcerdom</b> , og priste istedetfor denne det indre Lys , hvormed Christus umiddelbar fraoven oplyser det indvortes Menneske . For at erholde dette , sagde han , behsvede man ikke soeregne Theologer , ei heller vidtlsftige Boger eller Skoler ; thi Gud havde forjcettet , at han selv vilde Icere Alle og udgyde sin <b>Aand</b> over Gamle og Unge , Mcend og <b>Qvinder</b> . Kirker vare heller ikke nodvendige ; thi Gud boede ikke i Templer , byggede med Hoender . Hele den scedvanlige Maade at dyrke Gud paa var forkastelig , man skulde kun strcebe at levendegjsre det indre Lys , Christus , Guds Ord inden i os , det var Alt . Dette kunde ikke siee ved fcelleds Psalmesang, ei heller ved Prcediken eller sceregne kirkelige Skikke og Festdage , da alt Idre var unyttigt , men man maatte , naar man kom sammen blive siddende i andoegtig Taushed og vente , indtil Guds <b>Aand</b> drev En eller Anden til at tale ; da vilde Munden af sig selv fiyde over med , hvad Hjertet var fuldt af .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092103010'target=_>, 1863, Læsebog for Folkeskolen og Folkehjemmet</a><h4>740</h4>Naar vi betragtc Pattedyrenes store Familie og gjore os lidt noermere bekiendte med den Naturdrift , hvorefter dm nyfodte Afkom strax er istand til stiv at soge sin Ncerings Kilde , faa vi et overtydcndc Bevis for , hvor unaturlig ! det er , at Barnet stilles fra Moderen . Knapt er det lille Dyr kommen til Verden , fsrend det , drevet af en indre uimodstaaelig Tilskyndelse , med en forunderlig Sikkerhet » ssger og sinder Moderbrystet , der i rigeligt <b>Maal</b> tilbereder den Melk , som i Begyndelsen udgjsr dets eneste Noering . Den samme medfodte Drift har ogsaa det nyfsdtc Menneskebarn ; men det er mere hjcelpelsst end det unge Dyr og behovcr Bistand for at finde , hvad det fsger . Lykkeligt er det Barn , som faar fin fsrste Ncering af en sund <b>Moder</b> ; thi Modermclken lcegger den tryggeste Grundvold til et fundt Legeme , og nogen anden Fode behsver Barnet ikke i sterc Maanedcr . En god Melk er en nsiagtig Blanding af alle de enkelte Næringsemner , som under Fordsielscn med Lcthed danne et sundt Blod og af dette igjen alle Legemets Dele . Heri ligger da den simple og naturlige Forklaring paa Melkens store Betydning for Sundhed og Liv . Naar vi altsaa fporgc : hvorfor er Melk en faa stor Velsignelse som Næringsmiddel , ligger Svaret lige for os . Den er det , fordi den er den eneste af Naturen felv tilberedte Fsde , der samlet indeholder alt , hvad der er nsdvendigt , for at underholde en kraftig Livsvirksomhcdhosden unge Skabning . Naar desuagtet en anden Tro ytrer fig hos mange Folk , kan dette have sin Grund i Ukyndighed eller deri , at Melken er tynd og uncersom , eller den kan i nogle Tilfoelde <b>vise</b> sig utilstroekkelig , fordi den ei ydes Barnet i fuldt <b>Maal</b> . Tyr man under faadan Mangel til anden Melk og forn oftest til Kom eik , bliver Fslgen ofte den , at Barnet ikke trives og faar en uordentlig Fordsielse. Forklaringen herpaa er simpel og nyttig at kjende : Komelk er nemlig rigere paa Ost , fattigere paa Sukker , og derhos bliver den hurtigere fur end Kvindemelk . Naar nu Melken ved Mavesaftens Virkning

<h4>1878</h4>dan Strid om det Skjsnne , forn der i Almindelighed er om de sandselige Goder , som den ene stedse soger at tilrive sig paa den andens Bekostning . Det Skjsnne er derimod nctop stikket til at forene Menneskene og styrke deres Folelse af Fcellesstab . I Sandsen for det Skjonne msdes alle Mennester og det desto mere , jo mere de have af <b>Dannelse</b> og crgte menneskelig Fslelse . Hvor det skjonne Syn <b>moder</b> os , vcere sig i Natnrcn , i de menneskelige Trcek eller i de Omgivelser , et Menneske har vidst at berede sig . der foler enhver sig behagelig tiltrukken og venlig stemt . Menneskene aabne sig lettere for hinandcn og faa en Folelse af , at de netop som Mennesker ere bcslagtede . Vi knnne saaledes sige , at Sandscn for det Skjonne ikke alene minder os om , at vi have en <b>Aand</b> , men ogsaa om , at vi have een <b>Aand</b> , at der er noget i os , noget Hsiere og Wdlere , der ikke blot lilhsrer hver enkelt og stiller ham fra alle andre , men forn i Sandhed tilhsrer alle Mennesker , og som jeg og du har , ikke fordi vi ere jeg og du , men fordi vi ere Mennesker .

<h4>2243</h4>Men der er et andet Slags Mennesker , der ere saare agtvcerdige , ja hvem vi i meget maa onste at ligne , men som ogsaa ofte afsky Eventyr, Sagn og alt saadant . De . ere komne til en levende Bekymring for at blive salige ; de fornemme , at de ere arme Syndere , og ville saa gjerne faa sine Sjele frelste .  At vinde Christum og sindes i ham " — det er deres alvorligste <b>Formaal</b> . Ogsaa mange saadanne , as Guds Naade opvakte Sjele fordomme Eventyrene , kalde dem Verdens Tant og ville ikke , at Bsrn stulle hore dem . De have for en Del Ret : der gives mange , mange stygge Eventyr , som Bsrn stet ikke bsr hsre . De boere i sig Forgift , der hsilig kan skade , ja drcebe det Liv , Gud i Daaben har ftabt i et Barnehjerte . For de Loerde kunne saadanne Eventyr nok vcere nyttige at kjende , fordi de <b>vise</b> , hvor fordoervet det naturlige Menneskehjerte er , som gloeder sig i at opsinde slige fcele Digtninger til Morstab for sig felv og andre ; Bsrn derimod og Enfoldige maa paa det alvorligste sky dem . Men , naar mange for Salighedssagen Bekymrede fcelde den samme Dom over alle Eventyr og kalde dem alle skadelige og fordervelige for Bsrn , da have de neppe rigtig tcenkt over Tingen . Der gives mange , ikke blot morsomme , men gode og loererige Eventyr , som kunne sysselsatte Bsrns Sjele paa en oplivende og gavnlig Maade . Hvor morsomme de kunne voere , ved I selv , kjwrc Born ! Og det , som rigtig tiltaler eder og voekker eder til inderlig Deltagelse , det opliver ikke blot , det over ogsaa eders Sjelskrcefter , — og hvor gavnligt er ei det ? Men dernoest- ligger der ogsaa paa de gode Eventyrs Bund en Betragtning af Livet , som just bsr fsres ind i Barnehjertet ; ak , der kommer siden nok , som vil troenge denne Betragtning ud igjen af Sindet . I have vist lagt Moerke til , at i de gode Eventyr gaar det altid den Fromme og Forstandige vel , den Onde , Taabelige og Daarlige ilde ; det er ikke Fsdsel , Stand og udvortes Kaar , som gjor Udstaget . Ofte gaar det hel underligt til , faa at de uanseligste Ting , de stakkers Dyr , som den medlidende Gut hjoelver , et übetydcligt Troe , som den deltagende <b>Pige</b> tåger sig af , eller andet Lignende — siden ftelfer og bringer Lykke . Men saaledes gaar det , naar du ser med det rette Blik , ogsaa til i Verden : der fsrer ogsaa alt til Lykke og Gavn for den Fromme og Forstandige , til Skade og Fordoervelse for den Onde og Daarlige , om ikke altid eller strax i de udvortes Kaar , saa ganske vist i de indvortes og mod Enden . Og ofte , ja langt oftere , end vi toenke derpaa , bruger Gud det Smaa og Übetydelige til at fremlede Held og Lykke for Sine . Disse Sandheder , som mangt et Eventyr smukt og klart lcerer , kan du aldrig dybt nok indprente dig , kjoere Barn ! Din Lykke beror paa , at du ret eier dem . — I Eventyret er der intet Livlost ; alt lever , taler og handler . Se , saaledes er det i Grunden ogsaa i Skabningen omkring os , og Barnesiet ser ofte Livet der , hvor den Wldre med det stsvcre Blik stet ikke opdager det , og horer grant Talen der , hvor den 2 Eldre tykkes , alt er tyst og stumt . Har du ikke selv , kjoere Barn ! t . Ex . mange Gange seet , hvorledes

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014121208023'target=_>Rosenius, C.O., 1868, De ti Guds Bud</a><h4>545</h4>Ogsaa dette Vud har den Herre Christus forklaret ( Math . 7 — 32 ) ; og hvilken Naade er ikke dette ! thi ham maa vi dog hore , om vi ville blive salige . Ipderne havde handlet med det sjette Bud vaa samme Maade som med det femte : de saa blot vaa den grove Gjerning , som her var Wgtesiabsbrud thi det sjette Bud lyder egentlig saaledes: „ Du skal ikke bryde Wgteskabet , " og saa agtede de det for Intet , at Hjertet var fuldt af utilladelige Lyster og Begæringer , naar de blot kunde afholde sig fra disse virkelige UdMlse i Gjerning . Da kom den Herre Christus med denne dybe Forklaring : „ Jeg siger Eder , at hver den , som ser vaa en <b>Kvinde</b> for at begjcrre hende ( d . e . med Lyst og Forscrt til Synden ) , har allerede bedrevet Hor med hende i sit Hjerte . I Lyset af denne grundige Forklaring fkulle vi betragte dette Bud . Men lader os forst underscge , hvorledes det kan vcere at forståa , at uagtet dette Bud har en saa dyb <b>aandelig</b> Betydning , at det er ouertraadt blot med en uren Begjcrring og omfatter alle den urene Lysts Norelser og Ytringer , saavel i som udenfor Wgtestabet , ( hvilket mange Skriftsteder bevise ) , det dog egentlig lyder saaledes : " Du skal ikke bryde Wgtestabet . " Der ligger mulig en dyb <b>Lcerdom</b> angaaende dette Bud indesluttet heri . Ja , hermed

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006082100022'target=_>Sundt, Eilert, 1867, Om Husfliden i Norge</a><h4>3383</h4>Ord og udviste Vandel . Den Travelhet » , som <b>Kvinden</b> stedse har befundet sig i , har vist ogsaa varet til stort Gavn baade for at vedligeholde Velstand og hoeve hende i <b>kvindelig</b> Huslighed og Dyd som god Bsrneopdrager , hvilket jo er kommet den hele Sloegt inden Bygden tilgode . Men det maa ogsaa virkelig erkjendes , at hun i denne Tid ogsaa har det altfor travelt , da hun ved de mange Arbeider hindres baade i legemlig og <b>aandelig</b> <b>Uddannelse</b> . I de sidste Aar har man faaet en Fristole i Gang for de konfirmerede Drenge inden Gjoesdals Sogn , som holdes i 4 Z , 6 Uger aarlig . Men for <b>Piger</b> har man endnu intet Saadant gjort og kan heller ikke tcenke paa noget , da de siet ikke har Tid . Det er saaledes kun faa <b>Piger</b> , der komme udenfor sine egne Dore for at lcere noget , og det eneste , som disse faa loere , er Syning af Klceder , dog med et Tilfcelde undtagen, som var , at en <b>Pige</b> var i Dsteriloere Aar , hvilken <b>Dannelse</b> hun ogsaa senere har benyttet til Nytte for sig og fit Hus * ) . At <b>Kvinden</b> ogsaa hindres i legemlig <b>Uddannelse</b> , maa antages deraf , at gamle Folk siger her ere flere sygelige <b>Piger</b> nu end fsr . Vel er det just ikke den almindeiige Mening , at dette siulde have sin Grund i Spindingen eller andet Arbeide, men det ansees heller som en Folge af Modesygen og Bekladningen m . m . Og fijsnt dette for en Del ikke er saa ugrundet , saa tror jeg dog at maatte tillcegge Spindingen lige saa stor medvirkende , om ikke sterre Aarsag , eftersom den er saa vedholdende fra den tidlige Morgen til den sildige Aften i den stsrste Del

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014071808060'target=_>Bang, Herman, 1879, Realisme og Realister</a><h4>452</h4>« Mogens » er ganske vist født i dette Aarhundrede , men han er dog ikke noget Barn af vor Tid , der er reflekteret i sin Handling og rafhneret i sine Nydelser . Han staar udenfor Tiden . Med et Magtsprog og hvad der er lidet realistisk uden egenlig at forklare os , hvorledes det kan være gaaet til , er Amtmandens Søn holdt udenfor , jeg vil ikke sige sin Tids <b>Dannelse</b> , thi det kan tænkes , men helt udenfor sin Tids <b>Tankeliv</b> og Strømninger , hvad der for en Amtmands-Søn er et noget voveligt Experiment , fordi en saadan , nåar han blot er nogenlunde normal, altid af selve Luften næsten , af hørte Samtaler , afopfangede Brudstykker , vil komme baade Formen og Normen adskilligt nærmere , end « Mogens » kommer . Nu er Mogens egenlig en eventyrlig Figur , eller rettere hans Skikkelse er en tænkt Ramme om noget psykologisk Sandt . Han er en Mand , der søger en <b>Kvinde</b> , ligesom Marie Grubbe er en <b>Kvinde</b> , der søger en Mand . Jeg har andetsteds i denne Bog i Omtalen af « Fra Studiebogen:, - talt en Del om det første Indtryk , der afføder en Forfatters Værker , den første Spire , der skaber en større Produktion, og jeg har paastaaet , at man , nåar man kunde faa fat paa disse første Spirer , vilde naa meget langt ind paa Livet af noget centralt i Forfatterens Personlighed . Gjennem begge Jakobsens Arbejder - - « et Skud i Taagen » er maaske saa at sige Begyndelsen til hans næste Bog , ligesom « Mogens . , var det til « Marie Grubbe » - - gaar den samme Tanke : en Higen efter at sé sit eget Væsen aandeligt og legemligt suppleret med den

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016062248026'target=_>Stowe, Harriet Beecher, 1877, Mesterens Fodspor</a><h4>879</h4>som vi skulde formode , en guddommelig Lærer vikle vælge , ikke en i <b>aandelig</b> Henseende fremragende eller en meget god <b>Kvinde</b> , men tvertimod en , hvis Liv var anstødeligt , hvis Sæder var slappe , og hvis Aandsudvikling og <b>Dannelse</b> var ringe . Denne <b>Kvindes</b> Historie , den Maade , hvorpaa han søgte hendes Bekjendtskab , lagde Beslag paa hendes Opmærksomhed , banede sig Vei til hendes Hjerte og gjorde hende til en Missionær , som kunde lede Folkets Opmærksomhed hen paa ham , er paa en vidunderlig Maade fremstillet af Johannes . Vi har Billedet af en <b>Kvinde</b> — saadan som mange er , selskabskjær , godmodig , snaksom og blottet for høiere moralsk Følelse — , som nærmer sig til Brønden , hvor hun opdager den trætte Jøde , som har sat sig ned foråt hvile . Han baner sig Vei til hendes Bekjendtskab ved at bede hende om en Tjeneste , — den bedste Fremgangsmaade , hvor det gjælder at aabne Hjertet hos en <b>Kvinde</b> af hendes Natur og Stilling i Samfundet. Hun bliver aabenbart forundret over , at han vil tale til hende , da han er Jøde , og hun Datter af en foragtet og forhadt Stamme . „Hvorledes beder du , “ siger hun , „ som er en Jøde , mig , som er en samaritansk <b>Kvinde</b> , om Drikke ? " Jesus svarer hende i det billedlige og ophøiede Sprog , som var ham eiendomraeligt : „ Dersom du kjendte Guds Gave , og hvem den er , som siger til dig : Giv mig at drikke ! da bad du ham , og han gav dig levende Vand . " <b>Kvinden</b> ser heri kun en Leilighed til at anbringe et kvikt Svar : „ Herre , du har jo Intet at drage op med , og Brønden er 9

<h4>1216</h4>Men imidlertid inaa vi erindre , at den Herre Jesus lidt efter lidt dannede omkring sig , som det personlige Midtpunkt , et Samfund af Disciple , baade Mænd og <b>Kvinder</b> . Denne Discipelskare kan betragtes som Spiren til den kristelige Kirke , og da dette Samfunds Forholde er blevne ordnede umiddelbart af Frelseren seiv , har vi her den bedste Kilde til <b>Kundskab</b> om den <b>Aand</b> og det Princip , — skjønt ikke netop nødvendigvis den Form — , der bør gjøre sig gjældende ved alle Kirkers Organisation .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010011103010'target=_>Grundtvig, N.F.S., 1832, Nordens Mythologi, eller Sindbilled-Sprog</a><h4>664</h4>fiint det end nakkes . I <b>aandelig</b> Forstand havde vi alle tabt / / o ^ eH / , der ei uden Foie Kaldes den synderligste Part , og vore Hoveder var udenlands Kommet til at sidde paa fremmede Halse , saa H / - Kunde Att ^ ei / og / ^ ' e / ^ / e Kun daarlig forliges ; tlri vi maae huske , at hverken vare vore Fedre Menneske- Mdere , der gjorde reent Bord , hvor de satte dem ned , heller ikke har ret Mange af ViKingerne fort Koner med dem , men har giftet sig , hvor de Kom nen , og neppe fundet de fremmede <b>Kvinder</b> saa letnemme til at fatte deres <b>Aand</b> , som til at forstaae deres ' Mening , vet Universal-UistorisKe Sporgs- <b>Maal</b> var altsaa : om de Nordiske Hoveder udenlands etterhaanden Kunde omskabe de fremmede Hjerter , eller om Nordens <b>Aand</b> , for den udvandrede , havde gjort saa dybt et IndtryK paa Hjerterne herhjemme , at Hovederne igien paa en Maade Kunde voxe ud deraf ; men hvilken af velene der end skulde skee , indseer Man let , det maatte have Tid , og det Sidste iser Klinger saa eventyrligt , at det maatte virkelig skee , for det Kunde synes mueligt * ) .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008111103038'target=_>Becker, Karl Frederik, 1852, Karl Frederik Becker's Verdenshistorie</a><h4>879</h4>Imedens disse Ting sysselsatte alle Gemytter , forlangte en begeistret <b>Qvinde</b> fra Hertfordshire at stedes for det militcere Raad , hvor hun talte meget om Aabenbaringer , hun havde havt , og forsikkrede , at den Vei , man havde valgt , var efter Guds eget Vidnesbyrd den rigtige . Dette trostede og beroligede Mange , der hidtil endnu havde tvivlet . Kongen blev hentet fra Hurst og bragt forst til Windsor , derpaa til Whitehall . Skiondt han var Fange , var han bidtil bleven behandlet med alle de udvortes Wresbeviisninger , der tilkom hans hoie Rang ; at dette pludselig ophorte , gjorde et dybere Indtryk paa hans Stemning , end man skulde have ventet . Hidtil havde han havt Tid til at forberede sig paa Alt , og om han endog bestandigt endnu havde nceret noget Haab , var han dog fattet paa det Vcerste , hvilket de Tanker <b>vise</b> , han nedskrev paa denne Tid .  Doden beholder sit Skrekkelige , hedder det iblandt Andet , hvad enten jeg skal lide den ved et hurtigt Mord eller med hiin storre Formalitet , som mine Fiender ville tilfoie , for at udbrede et Skin af Ret over en saavel efter guddommelige som menneskelige Love fordommelig Gierning . De ville aldrig troe , at deres Verk vil kunne trives , forend det er blevet våndet med Kongeblod . Men Gud vil ikke lade dem lenge blive boende i det Babel , de have opfort vaa mine Veen og befestet med mit Blod . De ville midt iblandt dem selv finde mine Hevnere . Nu sunes mine Fienders Villie at vere den eneste Lov , deres Magt <b>Maalet</b> . deres Resultat Prøvestenen for hvad de kalde Retferdighed ; de forvente Sikkerhed ved min Dod og glemme , at netop da vil den strengeste Giengieldelse bryde los , naar det tilstedes Menneskene til det Uderste at giennemfore deres ugudelige Hensigter . Herrens Villie har temmet min , derfor vil jeg ogsaa doe uden Hevngjerrighet » , som det sommer sig en Christen

<h4>1273</h4>I disse Hoffets Sale og Haver uddannede sig hiin friere sclffabelige Tone , hiin levende og aandrige Conversation , hiint lette og behagelige Vesen , der blev et Monster for alle andre Hoffer , ja for alle Europas hoiere Stender , og ved denne sociale <b>Dannelse</b> , ved de fra det udgaaede <b>Moder</b> i Ord , Forhold og Seder fik Frankrig en ikke ringere Indstydelse end ved sine Vaaben og sine diplomatiske Forbindelser , da netop de betydeligste Mend ide ovrige Stater bleve berorte af disse Virkninger . Versailles og Paris bleve Verdens Hovedsteder . Den ved dette Hof forsamlede Adels fortrinligste Interesse var de utallige Intriguer om Anseelse , Indstydelse , Titler , Belønninger , Rang , Wresposter og Kongens Gunst og jevnsides disse Bestrebelser Galanteriets Tillokkelser . Den store Indstydelse, <b>qvindelig</b> Skionhed , <b>qvindelig</b> <b>Aand</b> og <b>qvindelige</b> Renker have

<h4>1555</h4>En fortreffelig Mand , Philip Jacob Spener — fodt i Rappolsweiler i Bvre-Elsasi den 13 de Januar 1635 , dod som Provst og Consistorialraad i Berlin den ste Februar 1705 — modarbeidede paa samme Tid med et velsignelsesrigt Resultat den store Fordervelse i den lutherske Kirke , som vi allerede have omtalt . Alle hans Bestrebelser gik ud paa at besne Theologien og det kirkelige Liv fra hiint dodeFormog Bogstavvesen , som hos de Fleste deugang gialdt for Christendommens Vesens , og vekke et fromt Sindelag , der kunde giennemtrenge hele Livet med en praktist-religios Streben og Handlen . Med en virkelig beundringsverdig, utrettelig Virksomhcd , som han uafbrudt fortsatte til sit Livs sidste Tid , virkedc han ved Underviisning , Predikener og Skrifter til dette <b>Maal</b> , og hans eget Liv fremstillede et folgeverdigt E ^ rempel paa hiin virksomme Christendom , til hvilken han saa inderlig onskede at lede Alle . Han var en deeltagende Ven , en Fader for de Fattige og Trengende, der aldrig gik utrostede fra ham , en mild Herre mod sine Undergivne, en trofast velvillig Collega for sine Embedsbrodre , et Monster paa huuslige Dyder i Omgang med sin <b>Kone</b> og i en alvorlig og dog kierlig Vorneopdragelse * ) . Den gavnlige Bevegelse i Gemytterne , han bevirkede ved sine Lerdomme og Formaninger , var saa stor , at han med Rette blev betragtet som en ny Reformator i den lutherske Kirke , der skrantede af dyd Brost . Han onskede , at Vandel og Seder igien maatte nerme sig de forste Christnes simple , bramfrie , afholdende Liv . Nogle af hans Tilhengere i Leipzig , til hvilke horte den senere saa beromte Magister August Hermann Franke , og deres Disciple udmerkede sig ved et alvorligt , stille og afsondret Liv , men troede , at de ogsaa udvortes i Klededragt , Miner og Geberder maatte legge den nye <b>Aand</b> for Dagen , som besjelede dem . De afholdt sig med selsom Overdrivelse fra mange i og for sig aldeles ikke syndige legemlige og aandige Nydelser , saasom Dands , Theatret , Sftog , Selskaber , Kortspil o . s . v. og paadroge sig derved Sp.Menavnet Pietister . Formedelst deres verdigere og dyberc Bestrebelser bleve de forfulgte og fiendligt behandlede af de Orthodore af det sedvanlige Slags , som ikke havde Sands derfor . Man lagde dem Vildfarelser i Troen og i Leren til Last , og dermed begyndte der en ny Kamp i den lutherske Kirke , de saa kaldte  pietistiske Stridigheder " . Spener selv blev derved <b>Maalet</b> for yderst heftige og lidenffabelige Angreb , men viste ogsaa ved denne Leilighed hiin Maadeholdenhed , Blidhed , Sjelero og Aandsmunterhed , der saa fortrinlig udmerkede ham i hans hele Adferd . Hvad man hos Spener kunde kalde Pietisme , var , som hans aandrige Biograph siger , , > noget

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092203020'target=_>Sommerfelt, Halfdan E., 1865, norske Zulumission</a><h4>1449</h4>daarligen mene , at de kun da kunne forkynde og prise Frelserens Navn , naar de optrcede som Ordets Prcedikanter , og derfor agte det som et Baand eller en ringe Gjerning at blive ydmygt troende , med christelig Nenhed og gavnrig Virksomhet » i det jordiske Kald , hvori de ere kaldte ! Nej , Brødre , denne Gjerning , Eders Gjerning er ikke ringe , thi den kan kun fuldbyrdes i en levende Tro og stal vcere en Aabenbarelse i alle Livsforhold af Apostelens herlige Fortrøstning , at vor Tro er den Sejer , som har ouervundet Verden ! I , kjcere Udsendinge , stulle lcere Hedningerne at kjende det christelige Familieliv ; hvorledes det forener Mand og <b>Kvinde</b> i gjensidig Hsjagtelse og selvfornegtende Velvilje , hvorved det udbreder en rig Velsignelse af jordisk Velvcere , Tilfredshed og Gudhengivenhed . I stulle <b>vise</b> dem at bruge Tiden og kjobe den belejlige Tid , saa at al nyttig Gjerning kan forfremmes og et christeligt Samfundsliv i stedse stigende Udvikling befordres , thi Guds Godheds Gaver ere mangfoldige , og Flittighed behewes . Men i denne Flittighed , i at bruge Verden , stulle I og lcere Hedningerne, hvorledes det er ikke at misbruge Verden , saa at Faderens rige Velsignelser ikke stulle anvendes til Ncering for syndige Begjæringer , men brugte i Nenhed og Wjsomhed stifte Gavn og Glcede i Mennestekjcerlighedens Tjeneste , at hans Kjcerlighed med Lovsange maatte prises , han , som elskede os felrst ! la , I stulle aabenbare for Hedningerne , hvorledes det christelige Hjerte i Tidens og Jordlivets Omskiftelser ikke ststter sig paa de Ting , som ere i Verden , men paa Guds grundfaste Barmhjertighet ) i Christo lefu og den evige Frelse i ham , saa at det ikke bortkaster sin Frimodighed , men med Haabet og Takken til ham stille og virksom gaar den Vej , hvorpaa han kalder . Dette er Eders Missionsgjerning , kjcere Brsdre , og den er ikke ringe , den er scerdeles stor , thi I ville vide det og kjende det tydeligere paa Hjertet , at den ikke lader sig fuldbyrde uden i Troen paa hans naadige Bistand , som har forjcettet:  leg ? r kommen for at gjsre alle Ting nye " , og som har givet Fortrsstningen :  Min Kraft fuldkommes i Skrsbelighed." Kjstber derfor <b>Visdom</b> , mine Brødre , at I stadigen maatte voxe i vor Herres Jesu Christi Naade og Kunostab , paa det at hvert Ord , I tale , maa blive talt til Herrens

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100603018'target=_>Baggesen, Jens, 1846, Jens Baggesens danske Værker</a><h4>7694</h4>> Volen , som nu brsd frem i Vstens straaleude Purpur Over den mulmafslsrede Jords Lyngstade , paa hvilken Lammet , bclcesset med Ulvenes Synd , blindt vandrcde fremad , Udstodt brat i den vide , dets Fjed vildftemmede Verden , Styrkede lidt den uskyldige Barm til at boere sin Vyrde . Nu forst saae Daglyset hans Blik i hele dets Omfang , Ei blot over sig dryppende ned med sparlige Draaber ISkovmorket , men vceldende rundt omkring ham , etGlarhav Druknende hele Naturen i Lyst . Den venlige Morgen Tog den af Msikhedens <b>Aand</b> med alle Rcedselens Kraller Grebne forvildede Flygtning i Favn , og lcegede Hjertets Giftigste blodige Såar med sit sandsfortryllende Sollys . Vel udsluktc det himmclsie Lys ei Helvedes tcendte Brand i hans blodhevntorstige Bryst ; vel vendte Naturens Hele Forvandling med Eet ei Naalen , der pegede nordpaa Dybt i hans Inderste , medens han gik i Dagen omkring ham Blindt til <b>Maalct</b> ; vel higede han bestandig mod Ictlund , Lydig sin <b>Moder</b> , og tro sin Samvittighed , bundet af Eden , Og af en skjult almcrgtig Magnet ustandselig hentraktj Mcn , hvad tilforn var Fortvivlelse , blev forvandlet til frit

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008052603001'target=_>Grønberg, B.C., 1836, Tydsk-dansk og dansk-tydsk Haand-Ordbog</a><h4>1894</h4>; w ! schen Sott und den Menschen , Midleren ; . ( hos nogle Hoondverlen ) Mellcmting mellem Svend og Dreng . Mittlerweile og- ; eit , av . d.T . imidlertid. Mittschiffs , av . midtstibs . Mittwoche , st-n , ( Mittwoch , m . - 03 - e ) Dnsdog , si . ; - wochentlich , aj . som sedvonlig steer om Dnsdogen , el . stecr hoer Dnsdog . Mitunter , s . Unter . Mitursache , k . > n , Medaorsag , e . f . - en ) Medbeiler ( - inde ) , Eoncurrent , Rival , « . Mitwirken , v. n . m . h . medvirke ; ( om Gud ) forundc de nedvendige Krefter paa en vedvarende Maode ; - kung , st - en . Medvirkning , si . , Mitwissen , li : - 3 ud . pl . Medviden » hed , e . ; - sser , m . - e pi . si « ss . Medriver, si . Mirtur , st - en , Mixiur , e . Mnemonik , st ud . pl . Mnemonik , Hukommclseskonst , Erindringskonst , e . Model , n . - s - n , Mebcl , » . Mobilien ( 4 Stov . ) pi . ud . « ss . d . T . Mebler , Lesere , rerligt Gods . Mode , t ' . - n . Mode , si . Mohe , av . efter Moden , sedvanlig , brugtlig ; - <b>dame</b> , t . - n < Modedame , si . ; - narr , m ^ - en - en , Modenar , si . ; - suchst l . ud . pi . Modesygc , o . ; - suchtig , aj . modesyg . Mlidel , m . - s pl . si « ss . et <b>Maal</b> « ed en Seileorden ; en Figup , et Nillede ( hos Syersker cg Veven ) ; en Stebeform. Modsill , n . - es - e , Model , < : . ; - iren , v. 1 ) i , , m . h . modellera . 2 ) 3. affor » mc ; - kunst , tt ud . pl . Modelleerkonst , si . ; - macher , m . - s pl . si « ss . Modellerer, « . Modeln , v. a . og n . m . h . die 3 euge , das Band - , forsyne med Figurer ; forme , dorme , give en <b>vis</b> Skikkelse . Formsterer , si . Modeltuch , n . - es-ucher , Novncklud . <b>Moder</b> , m . - 3 ud . pi . Mudder , Dynd . Moderig , aj . muddret ; som smoger cl . lugter rooddcnt . Mudern , v. n . m . h > roodnc , gooe i Forrooldnelsc ... Modsrn , aj . modem , pao Moden , efter den nycstc Mode . Modisch , aj . modens , pao , ester Moden ; iogttogende Moden . Mogen , v. i . n . m . h . moattte , kunne , ville , monne , gide : du magst ( maoe ) dich in Acht nehmen ; wie mag ( kon , monnt ) da3 ; ugehen ? er hatte es immer thun - ( for gemocht ) ; graden mag ( gider ) ich nicht ; mochte er doch kommen, gid hon dog vilde komme ; ich

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012070608275'target=_>Krummacher, F.W., 1846, Elisa</a><h4>1071</h4>Elisas Bud ankommer til Samaria . iler til Paladset , fremstillor sig for Kongen og udforer sit Wrende . — loram horer hans Budskab , ned Fornoielse ; han ster sig derved befriet fra en hemmelig Forlegenhed , og dette lader ham let for- Vinde det Bittre i Elisas Ord . Han lader den syge Feltherre sige , at i lericho boer en Prophet ; til ham maatte han umage sig hen . Aldrig gjorde en Veiviser af Trcee en mcre foronsket Tjeneste . end denne . Naamans Glcrde over denne Uventede Efterretning er übeskrivelig ; Vudskabet lyder for ham , som den sig ncermende Nedningsmands Aareflag lyder for en Skibbruden imellem Bolgernes Brusen ; som den frelsende Stemme, der raaber paa hans Navn , lyder for En , der er inde . spcrrret i en dyb Skakt . " Om en Prophet " , tcrnker han , "talede jo ogsaa den lille <b>Pige</b> i Tamascus ! " og han bliver ncesten til Mode , som hine <b>Vise</b> fra Osterland maae vcrre blevne , da de ved deres Udgang af Jerusalems Porte atter saae sig lyste af hiin Stjerne , der skulde lede dem til <b>Maalet</b> for al deres Lcrngscl , alt deres Haab . Hans sode Haabs allcrcde nedtraadte , og sorgeligt henvisnede Skabning er ved dette gode Budskabs milde Foraarsregn , i et Oieblik atter opblomstret i den stjonneste Glands , og hans Tillid til en ncrr Redning er hlevcn vissere og gladere , end den nogensinde for har vcrret . Ak , paa enhver Vandring , den gaae hvorhen den vil , kommer det kun derpaa an , at Gud bod os drage , og at Han såtte os i Vevcrgelse ; da kunne vi ikke forfeile <b>Maalet</b> . Vi Alle , saa Mange af os , som hore Ehristo til , ere paa Vandring , ligesom Maman , og ikke alene til hiin Stad , hvortil der var en Grund ; men til cndnu mange andre <b>Maal</b> af mange Slags . Vi Prcrdikanter

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014012426004'target=_>Holberg, Ludvig, 1860, Udvalgte Skrifter</a><h4>1008</h4>Peer tallene . ) Det er et underligt Slags Folk , disse Poeter . Om en anden crrlig Mand giver et Drefigen udi Tanker , trcrtter man ham til Daarekisten ; men hos dem er saadant Tegn til <b>Lcerdom</b> og Dygtighed . Ellers sik jeg det Vers for godt Kisb . De Folk maae have nogen Frihed , efterdi de ere saa lidet interesserede ; thi hvad er dog tolv Skilling for saadan lcrrd Mand ? Jeg forstaaer ikke Verset , men kan dog see , at det er altid sine Penge vcerd . Nu stal jeg bilde min Husbond ind , at jeg har givet to Nirdaler derfor , saa vinder jeg ved dette Wrinde elleve Mark og fire Skilling . Jeg har kun liden <b>vis</b> Lsn , maa derfor ncrre mig redelig ved Commissioner. Mangen brav Mand fsder <b>Kone</b> og Vom derved udi Kwbenhavn ; thi hvoraf skulde de gode Folk ellers leve ? Hvis jeg kunde flattere , som Jesper gisr , for min Herre , kunde jeg have de Smauser og Fordringer ' som han faaer . Jeg skulde ikke rose mig selv ; men jeg er en crrlig Karl , der ikke kan sige andet , end jeg mener . Naar min Husbond begynder at tale om sin Manddom i den Brabantste Krig , tier Peer ganske still ; thi jeg veed , han har aldrig vceret paa den anden Side af Belt , end sige udi Brabant , som ligger nogle tusinde Mile herfra. Nei , Wrlighed varer dog lcrngst ; det hedder hos mig , som Hollænderne siger : Thue das Rccht nicht , und fly den Teussel . Kunde jeg saa vel Tydfl , som jeg kan Hollandsk , var Peer anden Mand , end han er ; thi jeg haver tidt hsrt Alexander Skolemester sige : hvo der kan Tydfl , og haver nogle Midler dertil , kan reise hele Europia igiennem . Men Fanden er udi dette Der , Du , Das , det kan man allene fast i fiorten Dage ikke lcere .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014102408022'target=_>Leonhardi, Gustav, 1876, Nat og Morgen</a><h4>682</h4>Over Markomannernes vilde , hedenske Folk , hvilke vare Romernes farlige og mcegtige Grcendsefiender ( i det nuvcrrende Bbhmen ) , herffede fire hundrede Aar efter Christus en Konge , som havde en <b>Hustru</b> ved Navn Fritigild . Hun var af et blidere og mildere Sindelag end de andre haarde tydsse <b>Kvinder</b> . Til hende kom der engang en Mand fra det fjerne Italien og fortalte hende Meget og Underligt om det , som han havde seet og hsrt paa sine Reiser i de fjerne Lande . Han berettede hende ogsaa om de hoie , stjsnne Kirker i Romernes Lande , om Chrisii Navn , som der var helligt , om hans Lcere og om de fromme Mcend , som forkyndte hans Ord . Efter lang Kamp havde nemlig Romerne lcert at indsee og erkjende , at Guds Rige er mere end dette Verdens Rige , og deres Keisere havde antaget den Christi Lcere , som de tidligere saa grusomt havde forfulgt . Hiin Mand fortalte ogsaa Dronningen om Ambrosius , Biskop i Matland. Denne Ambrosius gjaldt nemlig for en Ststte for og et uovervindeligt Bolvcerk i Kirken . Fra hans Lceber flod som en Strsm den begejstrede af Gud beaandede Tale , som intet Hjerte formaaede at modstaae ; og selv den mcegtige Keiser Theod osiu s havde bsiet sig for ham . Ligesom nu Ambrosius ' s Navn var hvit anseet i alle Rigets Dele , saa kom ogsaa nu hans Ry og <b>Kundskaben</b> om hans Lcere til Markomannernes Dronning . Da blev Fri ti g ild s Hjerte greben af en dyb Lcengsel efter at hsre mere af det , som Christus havde forkyndt , og paa hiin Mands Ord lod hun da af fra de hedenw Guder , som ffulde boe i Trcrer og Skove , og vendte sig derimod til den Gud , som vil tilbedes i <b>Aand</b> og Sand-

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015111908005'target=_>Marie, 1879, Ved egen Kraft?</a><h4>1200</h4>« <b>Kvindens</b> sædvanlige Kald , at blive <b>Hustru</b> og <b>Moder</b> , kan aldrig blive mit » , hviskede hun ; « jeg ved , at mit Hjerte aldrig mere vil have Rum for saadanne Eølelser . — — <b>Kvindens</b> Kald ? — — Men er da ikke <b>Kvindens</b> som Mandens Kald det , at virke der , hvor Gud hen viser dem ? Han har afstukket mange Yeie for os her i Livet , og han havde ikke gjort det saaledes, hvis hans Finger kun pegede i e n Retning . Og er det da saa , at <b>Kvindens</b> sædvanlige Kald altid har staaet for mig som mit Livs Bestemmelse ? Har der ikke i Dybet af min Sjæl hvilet en anelsesfuld Trang , . som atter og atter har hævet sin Stemme og tilraabt mig : « Nei , der ligger ikke dit Livs <b>Maal</b> ! Kom og følg mig , og jeg vil <b>vise</b> Dig en anden Vei , hvor Du ogsaa i Stilhed kan arbeide og virke til Guds Ære , i Ly af <b>Kvindelighedens</b> værnende Hegn » » .

<h4>1636</h4>( Skilling-Magazin ) . » Hendes Skisser vidne alle om en hos en begyndende Forfatter sj elden Klarhed angaaende Omfanget af hendes Talent , der gjør , at man læser dem med en Følelse af . Tryghed ; thi hun vover sig ikke udenfor det Omraade , hvor hun er hjemme . Der gaar gjennem det hele en sund og ægte <b>kvindelig</b> Livsopfatning , hvilket ikke udelukker et paa sine Steder endog satirisk Lune , og hendes Gjengivelse af Karakterer og Livsforhold vidner om et aarvaagent psychologisk Blik . « ( , \ \ aktaren . ) » — — Ett nytt arbete , hvilken i likhet med den fbrra bår witne om den hjertats sanna uppfatning af ratta grunden for Ijfwets forhallanden , utan hwilken dessa måste forefalla den askadande som ett planlost kaos ; och då forf. desutom blifwit begafwad med en skarp og klar iaktagelseformåga , i fbrening med ett godt og lifligt framstållningssatt , kunna wi icke annat ån anse dessa små bocker , bland hvilka och till wår kånnedom kommit en med titel : » I Tusmørket ® , som små perlor bland allt det myckna , som ur det litteråra hafwet i wåra dagar wråkes uppa bunken , d . a , boklådsdisken . « Af samlede Anmeldelser af « I Tusmørket » , « Gjennem Kamp » og » Fra min Fødeby » hidsættes ; ( The Scholastic World . ) « \ i henvende engelske Læseres Opmærksomhed paa disse Bøger , baade fordi vi saavidt muligt gjerne vilde være Talsmand for alt , hvad der kan tjene til at befordre god Opdragelse i vore Familier saavelsom i vore Skoler , og tillige fordi vi føle en stor Deltagelse for ethvert Lands literære Bestræbelser , naar <b>Maalet</b> for disse er Opdragelsen til et renere Familieliv . — Disse Bøger er os Vidnesbyrd om , at et Strøg paa en Naturstreng gjør den hele Verden til en Slægt . De <b>vise</b> os , at med Livets alvorlige Virkelighed maa Kampen føres i en og den samme <b>Aand</b> ; at for den inderlige Forening af en levende og urokkelig Tro maa der kjæmpes med det samme Alvor i Bønnen ; at Forstand og Hjerte ikke kan stilles tilfreds ved Forbindelser , der ikke omfatte mere end det Ydre , men at man maa søge andetsteds og dybere , maa søge det Oversandselige ° S hig 6 efter sedlere Følelser end de jordiske og sandselige . — ( Skandinaven . ) « Alle disse tre Bøger har fundet en særdeles velvillig Modtagelse af Pressen i de skandinaviske Lande og i England , og « Maries » Læsekreds bliver med hvert Aar større og større . Forf . lægger ved disse Bøger et betydeligt Forfattertalent for Dagen , og jeg føler mig overbevist om , at der er mange af mine Landsmænd i Amerika , som ville læse disse smukke Livsskildringer med stor Interesse » .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006081800025'target=_>Belsheim, J., 1880, Veiledning i Bibelens Historie med udførligere Oplysninger om det Nye Testamentes Bøger</a><h4>288</h4>( Matth . 15 , 22 flg . ; Mark . 7 , 24 flg . ) , med den romerske Hovedsmand ( Matth . 8 , 5 ) , med Grcekerne ( Joh . 12 , 20 flg . ) og med Pilatus under Forhoret . Det Nye Testamentes <b>Maal</b> er imidlertid ikke rent , klassisk grcest , men for de fleste Bsgers Vedkommende blandet med adMige hebraiske Ord og Vendinger , i flere Boger ogsaa med en Del latinske Ord , ligesom hos de senere grceske Skribenter Plutark og Dio Kassius . Man har sundet det underligt , at de , som streve det Nye Testamentes Boger , kunde skrive saa godt , da flere as dem oprindelig vare Fiskere . Men foruden , at alle havde den guddommelige Indgivelse , havde flere af dem megen <b>Dannelse</b> . Apostelen og Evangelisten Matthceus havde vceret Tolder i Kapernaum ( Matth . 9 , 9 ) , Apostelen og Evangelisten Johannes var enten fra Jerusalem eller var vel kjendt der , endog med Ypperstepresten , hans <b>Pige</b> , hans Tjener Malkus og dennes Frcende , og maa have havt Hus og Hjem der ( Joh . 18 , 10. 15 — 18. 26 ; 19 , 27 ; Mark . 14 , 66 ; Ap . Gj . 1 , 13. 14 ) , og maa altsaa kun af og til have vceret ved Genesarets So for at fiste ( Matth . 4 , 21 ) . Johannes Markus var ogsaa fra Jerusalem , hvor hans <b>Moder</b> havde Hus ( Ap . Gj . 12. 12 ) . Han og Apostelen Johannes kunde altsaa have den <b>Dannelse</b> , som var at faa i Hovedstaden . At den ikke var ringe , se vi af losefi Exempel . Lukas , som har skrevet Evangeliet , Apostlernes Gjerninger og maaske Hebrceerbrevet , var Lcege ( Kol . 4 , 14 ) og maa altsaa have havt mange <b>Kundskaber</b> , da Lcegerne paa den Tid ikke havde ringe <b>Uddannelse</b> . De stilledes i Klasse med Dvrighedspersoner, Talere og Filosofer . ( Se Digesterne I , tit . 18 , fr. 6 , § 7 ; 1 ^ tit . 4 , fr. 18 , § 30. ) Paulus var fodt i Tarsus , der var i stort Ry for <b>Dannelse</b> , havde medfsdt romersk Borgerret , var altsaa af mere velhavend Forceldre og var oplcert i lerufalem af Gamaliel ( Ap . Gj . 16 , 37. 38 ; 22 , 3. 22 — 28 ) . Petrus var en ualmindelig fyrig <b>Aand</b> og det var ikke underligere , at han forstod grcrfl , end hans Broder Andreas og hans Landsmand Filip ( Joh . 12 , 20 flg . ) , hvilke alle vare fra Betsaida ( Joh . 1 , 45 ) . Han maa f . Ex . have talt grcefl med den romerske Hovedsmand Kornelius ( Ap . Gj . 10 ) . Af lignende Grunde kunde vel ogsaa Jakob og Judas grcrst .

<h4>744</h4>Forveien ( Kol . 4 , 10. 11 ) opregnet nogle af sine Medarbeidere ,  som ere af Omskjcerelsen " , altsaa Isdekristne . Blandt dem regner han ikke Lukas , men blandt nogle andre , som han derpaa omtaler ( Kol . 4 , 12 — 14 ) . Lukas var altsaa fsdt Grceker , var Lcege og havde derfor megen <b>Dannelse</b> ( se S . 33 ) , hvad hans Skrifter til Overflod <b>vise</b> , da han skriver renere grcest end de andre Evangelister. Men om han ogsaa har vceret Maler , som en senere Fortcelling ( Niceforus Kallisti i 14 de Aarh . ) gjor ham til , er uvist . Efter Ensebius ( Kirkehist . 111 , 4 ) , Hieronymus ( Fortalen til Lukas i Codex aureus ) o . fl. var Lukas fsdt i Antiochia i Syria . Det kan have Rimelighed for sig , og der kan han fsrste Gang vcere kommen sammen med Paulus ( Ap . Gj . 11 , 25 flg . ) . At han derimod skulde vcere den samme som Lucius af Cyrene ( Ap . Gj . 13 , 1 ) eller den Lucius , som omtales i Rom ( Rom . 16 , 21 ) som Pauli Frcende , det strider baade mod Navnedannelsen ( Lucas kan vcere forkortet af Lucanus , men ikke vcere det samme som Lucius ) og mod Lukas ' ydmyge Regel forsvrigt ikke at navngive fig selv i sine Skrifter . At Lukas , som nogle gamle Lcerere ( deriblandt et Origenes urettelig tillagt Skrift , Hippolht o . fl. ) have ment , var en af Jesu 70 Disciple ( Luk . 10 , 1 ) , er ikke urimeligt , og det hverken stadfcestes eller modsiges i Forordene til Evangeliet ( 1 , 2 ) , hvor der tales om dem , . . som fra Begyndelsen vare Oienvidner " . Man skulde ncesten tro , at denne Beretning er rigtig , saadant Kjenoskao som Lukas har havt til Isdeland og dets forskjellige Skikke , Styrelsesscet o . s . v. Han maatte i saa Fald have vceret en Portens Proselyt ( 2 Mos . 20 , 10 ) , da han , som sagt , var hedensk as Fsdsel , Han ved f . Ex . om Chusa , Herodes ' Foged i Kapernaum , hvis <b>Hustru</b> Johanna var blandt de <b>Kvinder</b> , som tjente Jesus med sit Gods ( Luk . 8 , 3 ) ; han ved om Manaen , som var opfsdt med Herodes ( Ap . Gj . 13 , 1 ) . Nogle have ment , at den kongelige Embedsmand i Kapernaum , hvis Ssn Jesus helbredede ( Joh . 4 , 46 — 53 ) , var den omtalte Chusa . Han fortceller saa livagtigt om de evangeliske Tildragelser , at man maa tro , at han har alt fra Dienvidner eller og felv ofte har vceret Bienvidne . Han viser sig som en vel underrettet Historieskriver , som fortceller alt med Nsiagtighed. Han scetter alt til Tid og Sted . Saaledes : Kvirinius ( 2 , 2 ) , Johannes Dsberens Fremtrcedelse ( 3 , 1. 2 ) , Jesu Alder

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111501013'target=_>, 1857, Love, Anordninger, Tractater, Resolutioner, Kundgjørelser, Departementsskrivelser, Circulærer m.m. for Kongeriget Norge</a><h4>8092</h4>. en saadan videre gaaende Hjcelp og Bistand , som ved de heromhandlede bevilgede Midler er tilsigtet . Findcr Fattigkommissionen da Anledning til at virke noget til dette Oiemed , saa behandles Sagen ganske slialedes , som under Mr . K er sagt . Finder Fattiglommissionen derimod . at der ikke er Anledning til at foretage noget Mrligcre . efterat de Omstandigheder ere ophsrte , som foranledigede den midlertidige Underststtelse , saa bliver ogsaa derom snarest muligt , helst inden angjeldende Personers Bortreise fra Distriktet , gjennem vedkommende Amt og Stiftsdirektion til Departementet at afgive Beretning , ledsaget med fornsden Forklaring om de Grunde , der have ledet Fattigkommissionen z < 1 ) dersom nogen i et bestemt Fattigvesmsdistrikt hjemstavnsberettiget Person alligevel i Folge sine Liosforholde forsvrigt kan ansees at henhsre til det her omhandlede Slags Personer , og dersom Distriktets Fattigkommission efter en omhyggelig Underssgelse sinder , at der ved en noget videre gaacnde Underststtelse end den almindelige Fattigforssrgelse lan vere Haab omatfaa samme Person bragt md i en ordentlig Ncrringsvei og fast livsstilling, saa vil der efter Omstcrndighederne kunne tilstaaes nogct Bidrag til Bestridelse af de forsgede Omkostninger , Som Erempel paa Tilftrld ' ' . hvor et saadant Bidrag vil kunne ventes , anfsres . at en i Distriktet hjemstavnsberettiget Mandsperson har levet i Konkubinat med en ikke i Distriktet hjemstavnsberetliget <b>Kvinde</b> . og at de ved den paapegede Underststtelse kunne haabes at ville indtrede i ordentligtWgteskab og tåge fast Ophold i Distriktet , 4 ) Personer af det her omhandlede Slags staa , saa lange de nyde Underststtelse . i samme Forhold til Fattigkommissionerne som andre Fattige , der ere under deres Forssrgelse . Paa Grund af det Siregne ved disse Personers Forholde bsr dog Fattigkommissionerne efter Omstendighederne end videre ved scrregne Foranstaltninger fore et nsiaatiqt Tilsyn med dcm og vaage over , at det gode <b>Formaal</b> , man har tenkt sig for dem , bliver opnaaet . Ved hvert Aars lldgang have vedkommende Fattigkommissioner gjennem vedkommende Autoriteter at indsende til Departementet Indberetning om hoer af de paa denne Maade forssrgrde Personer , med Forklaring om deres ydrc Kaar og moralske Forhold i den forlsbne Tid , og de forandrede Foranstaltninger , som for Fremtiden kunne ansees for snskclige . Saadan Indberetning afgives ogsaa til andre Tider . naarOmstcrndighcder indtrasse . som bevirke , at Underststtelsen ber ydes paa anden Maade eller ganske ophsre . 5 ) Det er en Selvfslge , at de Tilsagn, som Departementet kan afgive om en i flere Aar varende Underststtelse , ville verre betingede af . at der i sin Tid ssecr fornyet Bevilgning af Storthinget til dette Oiemed Men om dette skal skee . vil rimeligvis vesentlig bero paa Udfaldet af disse fsrste Forsog i denne Rttning . og om disse stulle <b>vise</b> sig opmuntrende eller nedslående , vil i hsj Grad afhange af den Deeltagelse for Verket , som Fattigkommissionerne maatte udvise . Departementet fonner sig derfor med en talrig Almeenhed i en tillidsfuld Anmodning til Fattigkommissionerne om at skjenke denne paa een Gang statsklogc og retfcrrdige og menneskekjcrrlige Bestrabelse al den Opmcrrksomhed . den fortjener . Ved samme Lejlighed har Departementet ogsaa opfordret de Fattigkommissioner . som dertil maatte see sig i Stand , til inden ncrrvcrrende AarsUdgang at indsende til Departementet en Fortegnelse over de i Distriktet hjemmehørende eller bekjendte Folk af det omhandlede Slags ( Fantcfslgcr o . Desl . ) , med Angivelse saa vidt muligt af hver Persons l ) Navn , 2 ) Alder , 3 ) Familieforholde lom gift eller ugift , om fsdt af omvandrende Forcrldre o . s . v ) , 4 ) Haandtering .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112301026'target=_>Munch, P.A., 1876, Samlede Afhandlinger</a><h4>1315</h4>blive kaldte Grevinder eller Marquisinder : den fattige Romer legger da Titelen til — thi næsten alle romerske Adelsmænd dave slige Titler — og <b>Damen</b> skaffer Pengene : saaledes ere begge Parter tilfredsstillede . Uagtet de Franske i politisk Henseende have Magten her , synes de dog ikke paa langt nær i det 80 ci2is Liv at kunne damle op med de Engelske : dertil ere de i det Hele taget for raa og übehagelige . Al denne Beundring af det Fremmede kommer dog ej af Mangel paa Patriotisme , men at den v6m73en6e Zevi ^ tnes om at staa tilbage for de Fremmede i <b>aandelig</b> Udvikling , og overhoved af den Fortrykthedsfølelse under de bestaaende Forholde , der nedtynger enhver dens Sind , der har nogen Anelse om , hvad virkelig <b>Dannelse</b> vil sige . Uvilkaarligt føler en saadan Romer sig - han være forresten saa patriotisk og stolt , nan vil - ligeoverfor den kundskabsrige Fremmede som Mindremand og forfalder derfor let til at copiere den , der ved sin ydre Fremtræden især imponerer dem . Med den simple Mand er dette nu anderledes . Uvidende og uopvakt , har han neppe engang nogen Forestilling om de Fremmede uden som Væsener , der tale et andet Sprog , og som egentlig kun ere til for at skaffe de Indfødte Fortjeneste og Fordeel , ligemeget om retmessig eller uretmessig . Men det vilde dog være højst übilligt , om man deraf lod sig forlede til at bryde Staven over den simple Romers Charakter . Hans Fejl fremtræde vel magßkee især for den Fremmede i et unv336li3t Lys , da de sjelden yttre sig paa den Maade i mere civiliserede Lande ; men ved at betragte dem nøjere vil man dog finde , at de mestendeels have sin Rod i Vankundighed , Mangel paa Aandsudvikling og overhoved de særegne Forholde , under hvilke han lever . Thi f . Ex . den simple Romer tigger uden resfølelse, stjeler og bedrager , somoftest hvor og naar han finder Lejlighed . Men han gjør det aabenbart mere af Naivitet , og fordi han ikke ret indseer det Nedverdigende eller Uretmessige deri , end af Ondskab . Ej engang et Indbrudstyveri eller et Stratenrøveri behøver hos dam at forudsette virkelig Forvorpenhed. Det milde Klima og Landets Frugtbarhed medfører , at den simple Mand dar faa Fornødenheder for at leve , tdi dau behøver kun et lidet Skuur at sove i , saa kan Gaden eller den frie Mark forresten være hans Stue : for en Übetydelighed kan han kjøbe den Haandfuld Bønner , eller Løg , eller Salat , hvoraf dan lever . Og da uu derimod legemligt Arbejde i dette Land , hvor Dagen fordetmeste er heed , falder meget vanskeligere og

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008022710001'target=_>Wergeland, Henrik, 1844, Læsebog for den norske Ungdom</a><h4>4694</h4>i det Enkelte , saavidt et Mcnncstc formaacr ; og denne Bedsthcd kunde han have . Mit Syn er for kort til at over < zlue , min Stemme for svag til at skildre Heltens Bane mod kauren , Kongens og Etatsmandens mod slt orhoiede <b>Maal</b> , derfor dveler jeg helst ved dette Sidste : Han clstsde sine Folk . Historien har optaget og vil altid optagc hans Navn som et af de yppcrste , der er tegnet i dens Aarbogcr , og scrtte en vcrrdig og fortjcnt Bauta raa hans Grav ; men skulde der ikke vave en Bauta , som var ligesaa sijon , og som gav hiin for en stor Dcel sin egcntlige Vard og Betydning ? Det norste Hjerte ncvrcr dybe , stcrrke og varige Folelser ; - dette Hjerte stal vcvrc din Bautasteen , Kong Carl Johan ! og Erkjendtlighed og Kjcrrlighcd stulle der staae faste , som vare de udhuggcde i Fcrdrenelandets Fjelde . Han var en Fader for tvcndc broderlige Folk det indeslutter meget dette Ord , dog ei for mcget for hans store <b>Aand</b> ; men vi erkjende det jo ogsaa , og medens vi reise Mindekors over saa mangen Grav , der gjemmer vore Kjcrres SWv , over Fader og <b>Moder</b> , Ven og Viv og Born , ville vi ei glcmme at reise et Mindekors i vort Hjerte for den Drot , som clskede os og som vi elskcde . Hans Stov hviler ei i vor Fcrdrcnejord , men i Mindcts Hal vandrer hans faderlige , crrvcrrdigc , kraftige Gestalt for vort Die og vor Ihukommclse ; og h « n er ikke i merc levende Erindring hos Broderfolket end hos os .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081403014'target=_>, 1860, &lt;Uten tittel&gt;</a><h4>1209</h4>Peer caNme , ) Det er et underligt Slags Folk , disse Poeter . Om en anden crrlig Mand giver et Drefigen udi Tanker , trcrkker man ham til Daarckisten ; men hos dem er saadant Tegn til <b>Lcerdom</b> og Dygtighed . Ellers sik jeg det Vers for godt Kiob . - De Folk maae have nogen Frihed , efterdi de ere saa lidet interesserede ; thi hvad er dog tolv Skilling for saadan lcerd Mand ? Jeg forstaaer ikke Verset , men kan dog see , at det er altid sine Penge vcrrd . Nu stal jeg bilde min Husbond ind , at jeg har giver to Nixdaler derfor , saa vinder jeg ved dette Wrinde elleve Mark og fire Skilling . Jeg har kun liden <b>vis</b> Lon , maa derfor ncrre mig redelig ved Commissioner. Mangen brav Mand fsder <b>Kone</b> og Born derved udi Kiobenhavu ; thi hvoraf stulde de gode Folk ellers leve ? Hvis jeg kunde flattere , som lespcr gisr , for min Herre , kunde jeg have de Smauser og Forcrringer , som han faaer . Jeg skulde ikke roje mig selv ; men jeg er en crrlig Karl , der ikke kan sige andet , end jeg mener . Naar min Husbond begynder at tale om sin Manddom i den Brabantske Krig , tier Peer ganske still ; thi jeg veed , han har aldrig vcrret paa den anden Side af Belt , end sige udi Brabant , som ligger nogle tufinde Mile herfra. Nei , Wrlighed varer dog lcrngst ; det hedder hos mig , som Hollenderne sigeri Thue das Nccht nicht , und sty den Teuffel . Kunde jeg saa vel Tydsk , som jeg kan Hollandsk , var Peer anden Mand , end han er ; thi jeg haver tidt hort Alexander Skolemester sige : hvo der kan Tydst , og haver nogle Midler dertil , kan reise hele Europia igiennem . Men Fanden er udi dette Der , Du , Das , det kan man allene fast i fiorten Dage ikke wre .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100603014'target=_>Becker, Karl Friedrich, 1844, Friedrich Becker's Verdenshistorie</a><h4>956</h4>Imedens disse Ting sysselsatte alle Gemytter , forlangte en begeistret <b>Qvinde</b> fra Hartfordshire at stedes for det militære Nacid , hvor hun fatede meget om Aabenbaringer , hun havde havt , og forsittrede , at den Vei , man havde valgt , var efter Guds eget Vidnesbyrd den rigtige . Dette trøstede og beroligede Mange , der hidtil endnn havde tvivlet . Kongen blev hentet fra Hnrst og bragt forst til Windsor , derpaa til Whitehall . Skiondt han var Fange , var han hidtil bleven behandlet med alle de udvortes LEresbeviisninger , der tilkom hans hoie Rang ; at dette pludselig ophorte , gjorde et dybere Indtryf paa hans Stemning , end man siulde have ventet . Hidtil havde han havt Tid til at forberede stg paa Alt , og om han endog bestandigt endnu havde næret noget Haab , var han dog fattet paa det Værste , hvilket de Tanker <b>vise</b> , han nedskrev paa denne Tid . „ Doden beholder sit Strækkelige , hedder det blandt andet , hvad enten jeg stal lide den ved et hurtigt Mord , eller med hine storre Formaliteter , som mine Fiender ville tilfole , for at udbrede et Skin af Ret over en saavel efter gnddommelige som menneskelige Love fordømmelig Gierning . De ville aldrig troe , at deres Værk vil kuune trives , strend det er blevet våndet med Kongeblod . Men Gnd vil itte lade dem længe blive boende i det Babel , de have opfort paa mine Been og befæstet med mit Blod . De ville midt iblandt dem selv finde mine Hevnere . Nu synes mine Fienders Villie at være den eneste Lov , deres Magt <b>Maalet</b> , deres Resultat Provesteuen for hvad de kalde Retfærdighed ; de forvente Sitterhed ved min Dod , og glemme , at netop da vil den strengeste Giengieldelse bryde los , nåar det tilstedes Menneskene til det Uderste at giennemfore deres ugudelige Hensigter . Herrens Villie har tæmmet min , derfor vil jeg ogsaa doe uden Hevngierrighed , som det sommer sig en Christen

<h4>1356</h4>I disse Hoffets Sale og Haver uddcmnede sig hiin friere selstabelige Tone , hiin levende og aandrige Conversttion , hiint lette og behagelige Væsen , der blev et Monster for alle andre Hoffer , ja for atle Enropas hoiere Stænder , og ved denne sociale <b>Dannelse</b> , ved de fra det udgaaede <b>Moder</b> i Ord , Forhold og Sæder fik Frankrig en itte ringere Indflydelse end ved sine Vaaben og sine diplomatiske Forbindelser , da netop de betydeligste Mænd ide ovrige Stater bleve berorte af disse Virkninger . Versailles og Paris bleve Verdens Hovedstæder . Den ved dette Hof forsamlede Adels foltrinligste Interesse vare de utallige Intriguer om Anseelse , Indflydelse , Titler , Belønninger , Rang , Wresposter og Kongens Gunst og jævnsides disse Bcstræbelser Galanteriets Tilloffelser . Den store Indflydelse, <b>qvindelig</b> Skionhed , <b>qvindelig</b> <b>Aand</b> og <b>qvindelige</b> Nænfer have

<h4>1644</h4>En strtræffelig Mand , Philip Jacob Spener — fodt i Nappolsweiler i Ovre-Elstsi den 13 de Januar 1635 , dod som Provst og Consistorinlrnnd i Berlin den ste Februnr 1705 — modnrbeidede paa stmme Tid med et velsignelsesrigt Resultat den store Fordærvelse i den lutherske Kirke , som vi allerede hnve omtalt . Alle hans Vestræbelser gik nd paa at befrie Theologien og det kirkelige Liv fra hiint dode Form' og Bogstavvæsen , som hos de Fleste den Gang gialdt for Cl / ristendommens Væsen , og væffe et fromt Sindelag , der kllnde giennemlrænge hele Livet med en praktist-religios Stræben og Handlen . Med en virkelig beundringsværdig, utrættelig Virffomhed , som hau uafbrudt fortsatte til sit Livs sidste Tid , virfcde han ved Underøiisning , Prædifener og Skrifter til dette <b>Maal</b> , og hans eget Liv fremstillede et folgcværdigt Erempel paa hiin virfsomme Christendom , til hvilken han sta inderlig onstede at lede Alle . Han var en deeltagende Ven , en Fader for de Fattige og Trængende, der aldrig gik utrøstede fra ham , en mild Herre mod sine Undergivne, en trofast velvillig Collega for sine Embedsbrodre , et Monster paa hllnslige Dyder i Omgang med sin <b>Kone</b> og i en alvorlig og dog kiærlig Borneopdragelse * ) . Dell gavnlige Bevægelse i Gcmytterne , han frembragte ved sine Lærdomme og Formaninger , var sta stor , at han med Rette blev betragtet som en ny Reformator i den lutherske Kitte , der skrantede af dyb Brost . Han onstede , at Vandel og Sæder igien maatte nærme sig de forste Christncs simple , indgetogne , afholdciidc Liv . Nogle af hans Tilhængere i Leipzig , til hvilke horte den senere sta beromte Magister August Hermauu Franke , og deres Disciplc udmærkede sig ved et alvorligt , stille og cifsondret Liv , men troede , at de ogsta udvortes i Klædedragt , Miner og Gebærder maatte lægge den nye <b>Aand</b> for Dagen , som besjæledc dem . De cifholdt sig med selsom Overdrivelse fra mange i og for sig aldeles ikke syndige legemlige og nnndige Nydelser , snnsom Dnnds , Thecttret , . Spog , Sclstaber , Kortspil o . s . v. og paadroge sig derøed Svottenavnet Pietister . Formedelst deres værdigere og dybere Vestræbelser bleve de forfulgte og fiendtlig behandlede af de Orthodore af det sædvanlige Slags , som itte havde Sands derfor . Man lagde dem Vildstrelfer i Troen og i Læren til Last , og dermed begyndte en ny Kamp i den lutherske Kirke , de sta kaldte pietistiske Stridigheder . " Speuer selv blev derved <b>Maalct</b> for yderst heftige og lidenstabelige Angreb , men viste ogsta ved den Leilighet ) hiin Maadeholdenhed , Blidt / ed , Sjælero og Aandsmunterhed , der sta fortrinlig ndmærfede ham i hans hele Adfærd . Høad man hos Spener knnde falde Pietisme , var , som hans aandrige Biograph siger , „ noget

<h4>1978</h4>I Italien stndt Christina en glimrende Modtagelse . Alle de Stæder , igiennem hvilke hun tog Veien , vare festlig smykkede ; af Borgerne blev hnn overalt modtaget med Jubel , og atle Steder ventede hende Giestebnd , Skuespil , Illumincttioner og Wresporte ; hele hendes Reise fra Infpruck til Rom var et Triumphtog . I Rom selv havde man opbudt den holeste Pragt ( 21 de December 1655 ) . Hun kyssede Pave Alcrander VII . Fod , modtog hans Velsignelse og strev sig fra den Tid Christina Alercmdra . Hun styltede derpaa ind i det strncstste Palads , samlede snart et Hof af Lærde og Kunstnere om sig og forogede de Samlinger af Boger og Kunstskatte , hun havde bragt med sig fra sit Fædreneland . Men længe tilfredsstillede Livet i Rom hende itte . Hun vilde see Frankrig og tiltraadte i Sommeren 1656 Reisen derhen , endsiiondt hendes pecnniaire Omstændigheder vare sta dercmgerede , at hun forst maatte panlsætte Juveler , for at kunne skaffe de fornodne Penge tilveie . Hele Paris var i fpændt Forventning for at see dette Vidunder af et Frncntimmel . Mazarin sorgede for et prægtigt Indtog og en ærefuld Modtagelse . Ludvig XIV . , som den Gang itte var meer end 18 Aar , var meget undseelig , da han lod sig præsentere for hende . Imidlertid fik Hoffets <b>Damer</b> snart Mod igien , da de stae Dronningens hoie Skulder , hendes lille Udvortes , hendes siiodeslose Paaflædning og übetydelige Folge . De fandt , at hun til Forbauselse paalagde sig liden Tvang , at huu domte frit om Alt , loe hoit i Thecttret , i Selsiaber uudertiden endog lagde det ene Been over det andet , med mere stadcmt , der var en Dodssynd-for dette Hof , som var vant til paa det strengeste at overholde de udvortes Former . Huu gjorde imidlertid Alt , str at forsone <b>Damerne</b> med sig , idet hun imodekom dem med den storste Hostighed , sagde dem Smigrerier og altid nedstemmede Convcrsttionen efter deres Ideekreds . Mændeite af <b>Aand</b> og <b>Dannelse</b> fortryllede hun ved det beundringsværdige Omfang af sine Kundsiaber og ved den utvungne , tevende og indtrængende

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008090403001'target=_>, 1842, Kongelige Forordninger, aabne Breve og andre trykte Anordninger for Norge</a><h4>4521</h4>Blegne , og som viser sig strax efter en mere eller mindre kjendelig Sygdom , ansees som mistcrnkeligt for at verre Kopper , og bsr dcsaarsag anmeldes paa den l Z § 15 og 11 » befaledc Maade , under de deri fastsatte Straffe i Overtrcrdelses - Tilfcrlde , naar Sykdommen siden befindcs at verre Kopper . , 12 > - Enhvrr Huusfader eller <b>Huusmoder</b> stal , saasnart Tegn til Bsrnekoppcr <b>vise</b> sig i deres Huse , « fortollet gjsrc Anmeldelse derom , paa Landet for Stedets Prcrst eller Sognefogden , om han boer ncrrmere ved Stedet , hvilke igjen stal anmelde det for Districtslcrgcn og Politicmesteren , som strax har at indberette det til Amtmanden ; i Kjsbstcrderne for Byfogden eller Stedets Lcrge , og i Kjsbenhavn for Politiemestcren eller Stadsphysicus, paa det at de fornsdne Foranstaltninger mod Sygdommen strar kan vorde fsiede . Mangel af en saadan Anmeldelse paadrager den skyldige Huusfader eller <b>Huusmoder</b> Straf fra 8 Dages Fcrngsel paa Ljand og Brot , indtil 3 Maanedcrs Arbeide i Forbedringshuset efter Forseelsens Beskaffenhet» og Domstolenes Skjsn . Ill ) Ligeledes stal det og i Kjsbstcrderne vcere Pligt for enhver Huusvcrrt , eller den , som i hans Sted har Opsyn i Huset, noie at underrette sig om , og uopyoloeligen for Districtslcrgen at anmelde denne Sygdoms Udbrud . Besindes han at have efterladt saadan Anmeldelse , uagtct Sygdommen har vcrret ham bekjendt , straffes han med Mulct fra 1 U Ndlr . til IUU Ndlr . efter hans Vilkaar og Forseelsens Grad . Som Folge heraf berettiges Huusvcrrterne til enhver Underssgelse , hvorved de kan scrttes i Stand til at opfylde den dem paaliggende Pligt . 17 ) Saasnart Lcegerne , enten paa den i forncrvnte § Z befalede Maadc , eller og ved egen Erfaring komme til <b>Kundskab</b> om Koppcsygdommens Udbrud , stal de , forudcn at indberette det til Amtmanden , uopholdeligen give den vedkommende Stifts- eller Landphysicus saadant tilkjende ; ligesom de ogsaa i dette Tilfcrlde strax bsr gjsre Indberetning til Commissionen for Vaccinationen . 18 ) De Forsigtighedsregler , som ved Frr . 1 ? April 1782 for Danmark ' ) og 5 Septbr . 1 ? ! N for Norge ere foreskrevne i Henseende til andre tpidemiske Sygdomme , stal ogsaa verre udvidede til Koppesygdommen , forsaavidt ikke ved denne Anordning anderledes er bestemt . Disse Forsigtighedsregler ere : at ingen Uvedkommende maa indsinde sig hos de Syge ; at Ssrgestuer og alle andre Almue-Samqvem forbydes ; at de Dsde begraves inden 48 Timer ester Dsden , med det samme Tsi , hvori de ere bortdsde , og i Kister, som indvendigen ere begede eller tjcrrede , samt nedgraves 4 Alen under Jorden ; og endeligen at hverken Liigfslge eller Begravelses-Gilder tillades . Il ) ) Naar der til noget Sted i Rigerne indkomme Skibe , som enten have Koppc-Patienter , eller Klcrder , hooras disse have gjort Brug , inden Borde , bsr de henlcegges under Qvarantaine , saal « nge , indtil de Syge ere blevne afhentede . og disses Klcrdcr bchsrigen ere rsgede og rensede ombord , samt lige Forsigtighed er iagttaget ved det Num , hvori de Syge have opholdt sig . - > 3 V_H > aa det at Koppesygdommen ei ved den hidtil brugte Inoculation stal kunne vedligeholdcs , og dens farlige Smitte paa ny udbrede sig , forbydes herved paa det strengeste al videre Indpodning af Smaakoppernc eller de saakaldte Bsrnekopper . Skulde nogen Lcrge desuagtet driste sig til at foretage nogen Inoculation af disse Kopper , da stal det verre ham forbudet videre at practisere nogetsteds i Kongens Niger og Lande , og stal han derhos om han er Embedsmand , have det ham anbetroede Embede forbrudt . Ligeledes stal og Enhver , der enten paa sin egen Person , eller paa sine Bsrn eller Andre , lader foretage Indpodning af forommcldte Kopper, ansees som Kongl . Befalingers forscrtlige Overtrcrdere , og straffes , om han er Embedsmand , med Affcrttelse fra sit Embede , og i andet Fald med Pengebsder eller anden Straf efter Omstcrndighcderne . 21 ) Ligesom der for Kjsbenhavn er indrettet et Locale i det almindelige Hospital , hvori

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006081600046'target=_>Lyng, Ge. Vilh., 1867, Jødedommen</a><h4>610</h4>netop det , denne gammeltestamentlige Fortelling om Syndens Fodsel i Menneskelivet nu har vist os — at Lysten er Syndens <b>Moder</b> . Lysten er det , at Skabningen i sin Forblindelse sinder sit Livs Kilde og sin Villies <b>Maal</b> i sig selv , istedeifor i Gud . Men Skabningens Liv som Skabning er et Natmliv , et Liv i Rummet og Tiden og dersor underkastet Forkrcenkelighedens alveldige Lov , modsat det Evige , Gudslivet , hvori der  ikke er Forandring eller Skygge af Omskiftelse " . Det er ikke alene saa , at vi vide , at Alt i Skabningens Liv , alt Jordisk , ester hin Nodvendighedens Lov vil forgaa ; men denne Forkrcenkelighed ligger saa ganske i det Naturliges , det lordistes Vcesen , at selv den lordifksindede ikke kan enste det anderledes . Ogsaa han fsler , at selv lordelivets hsieste Nydelse vilde blive Ham modbydelig , vilde blive saa at sige et Vroengebillede af sig selv , dersom den vårede evig eller blot for lcenge , fordi den er det Ufuldkomne , fordi Mennesket kun tilfredsstilles ved det Fuldkomne , og fordi det Ufuldkomne stedse maa og stal vige Pladsen for det mindre Ufuldkomne , medens derimod Gudslivet som det Fuldkomne ikke har Noget at vige Pladsen for . At Skabningen vil have sit Liv i sig selv , vil altsaa sige det Samme , som at den foretrekker at have sit Liv i Dieblikket , i det Forgjengelige, fremfor i det Evige : Lysten , der griber Eva i hint store kritiske Dieblik , det er den Aandens forferdelige , men tillige vellystig kildrende Svimmel , hvori det <b>aandelig</b> sortner for Wiet , hvori Fortid og Fremtid forsvinder , og Aandens Die stirrer ufravendt med hele sit Syn vaa det Enkelte , igrunden kun et forsvindende Punkt , det hvori nu hele Livets Fylde concentrerer sig , nemlig Dieblikkets Nydelse . Men ligesom Solens Straaler , hvis Natur det

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015012808009'target=_>Hertzberg, Nils, 1880, Opdragelsen i Hjemmet</a><h4>246</h4>Hvad Forceldrene angaar , er det imidlertid en anden Sag ; mellem dem og deres Born er den naturlige gjensidige Hengivenhed saa stcerk , at et saadant Misforhold vel horer til de sjceldne Undtagelser ; hvad Forældrene angaar , kan det vel ogsaa med Sandhed siges , at , hvem Gud giver Embede , giver han ogsaa Forstand ; i den naturlige og af Kristendommens <b>Aand</b> lutrede og adlede Kjwrlighed har Fader og <b>Moder</b> en übedragelig Vejleder og en uudtommelig Kilde til opdragende <b>Visdom</b> ; under alle Omstændigheder er dog denne en guddommelig Naadegave , hvem det , ligesom enhver anden Naadegave , der har sin Indvielse og henter sin Kraft fra et personligt Gudsforhold , gjcelder at ncere og styrke ved samvittighedsfuldt Brug af Naadens Midler ,

<h4>672</h4>Endog de hedenske Romere holdt strengt over den Regel- „ Bsrn stylder man den stsrsteHensynsfuldhed " < iu9 , xiin3 , HkbOtnr Myr-in Samtalen maa i Borns Ncervcerelfe ikke blot vcere sig bevidst , at saadanne ere tilstede , men stal , om end dette i visse Maader kan virke hemmende van de Voznes Frihed , saavidt muligt gjsres frugtbringende for Bornenes Oftdragelfe. Der er en stjult Egoisme i , eller , der ligger en Fslelse af en mindre god Samvittighed til Grund for , at Mcrndene , naar de skulle hengive sig til den indbyrdes Samtales Glcede , <b>vise</b> Bornene vaa Dsr , ligesom det er en Mangel ikke alene vaa Ridderlighed , men ogsaa vaa sand og fin <b>Dannelse</b> , ja ofte et Mcerke paa Raahed , naar Mcendene under selskabeligt Samvcere sondre sig fra <b>Kvinderne</b> ; ligesom Bornenes Tilstedevcerelse uvilkaarlig bringe et velgjsrende Moment bande af Tugt og af frist Umiddelbarhed og Naturlighed ind i de Voxnes Samvcere , saaledes bringer dannede <b>Kvinders</b> Ncervcerelfe et Moment af Tugt og <b>Dannelse</b> ind i det fcelles Samvcere af Mcend og <b>Kvinder</b> ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111600013'target=_>Büchsel, Carl, 1877, Landsbypræsts Erindringer</a><h4>425</h4>lupsdagene som glade , festlige Dage , og der ligger oasaa en Berettigelse til Grund herfor . Dnsternes <b>Maal</b> er naaet , Kjerligheden haaber paa Lykke og gode Dage , og Haabet gjsr Hjertet glad . Men paa den anden Side maa man ogsaa ttlstaa , at der neppe gives nogen Dag , paa hvilken Lwet har et saa alvorlig ! Preg . Hvad en <b>Pige</b> foler , naar Moderen lcegger Brudekrandsen om hendes Pande , og hun forlader sit Fedrehjem og i Kirken gaar opad Alterets Trin for at udtale det lllle Ord , der binder hende til Manden , indtil Dsden stlller dem ad , og hvad Ynglingen tenker , naar han staar ved hendes Side for Guds Aasyn og rekker Haanden ud mod hende for aldrig mere at skppe hendes - det har Sproget intet Ord for . Hvad der rsrer sig i Foreldrenes Hjerter , naar de bereder sig til at stilles fra sit Barn og ser det staa for Alteret , det er meget alvorlige Tanker . Wgteskaoet er den celdste Indstiftelse , som gtves: det har Forjoettelser om den guddommelige Velsignelse og stal derfor fsres paa knstelig <b>Vis</b> for Gud Herrens Aasyn . Brudefolkene stal en < aana staa for Gud Herren paa Dommens Dag , den Enes Ro og Fred i Livet vil blive krevet af den Andens Hustruens Taarer , der paa Jorden kun er Vanddraaber , der let dunster bort , oa Mandens Sukke , som kun er en Lyd i Luften , vll paa hin Dag komme til at veie tungt . Det ene Menneske kan gjsre det andet meget ulykkekgt , oa ikke alle Brudefolk har i Wgtestabet fundet den Lykke , de har drsmt om og haabet paa . Paa den alllde Bryllupsdag har ofte fulgt meget ssrgeltge Daae . Om den legemlige Skjsnhed siger Sknften , oa Erfaringen stadfester det : Gresset visner , og Blomsteret oerpaa falder af . Den naturlige Kjerlighed,

<h4>444</h4>med Hensyn til <b>Dannelse</b> , Stand , Rang og Alder , og en <b>vis</b> Lighed bsr vcere tilstede mellem dem , som indtrcrder i LEgteskab . Den sande , oprigtige Tilbsielighed bestemmes af noget , som er meget vanskeligt at bestemme ; naar den engang er vaagnet , kan den overspringe mange Skranker , og übetmget kan man ikke dsmme derom . Fornuftige Forceldre maa under Bsn soge at udforske Herrens Villie og anraabe ham om at styre sit Barns Hjerte , men ogsaa erkjende , at Foreldrenes Magt virkelig har sine GrceMer , og at Forlovelsen , om den end er afhcengig af Forcrldrenes Samtykke , dog visselig ligger ganske ncrr denne Gromdse . Bsrnene maa have den bestemte Fslelse , at de selv maa vcere ansvarlige sor det Skridt , som i vcesentlig Grad lssner deres Forhold til Fcedrenehjemmet . Heraf fslger . at det idetmindste er meget betlrnkekigt at sve Tvang saavel til den ene som til den anden Side ; dette kan let give Anledning til Misforhold og Forviklinger af det vcerfte Slags , hvilket ogsaa Erfaringen tydeligt lcerer . Det er mere tilraadeligt at en yngre <b>Pige</b> crgter en crldre Mand end omvendt , thi Manden stal lede Hustruen og ikke Hustruen Manden , og den yngre har lettere for at underordne sig end den crldre . Ligesaa er et lykkeligt Wgteskab snarere tcrnkeligt , naar en rig Mand crgter en fattig <b>Pige</b> end omvendt . Det strider mod en dygtig Mands naturlige Fslelse at leve af Hustruens Formue , fordi han er forpligtet til at erncrre <b>Kone</b> og Bsrn . Meget vakkert og snskeligt er det ogsaa , naar begge Parters Familie staar i et venskabeligt Forhold til hverandre , og det er idetmindste deres Pligt for de unge Wgtefolks Skyld omhyggelig !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009090903009'target=_>Sundt, Eilert, 1859, Om Ædrueligheds-Tilstanden i Norge</a><h4>2953</h4>Imidlertid , som bel overalt i Landet , saaledes kan der ogsaa i Heggebostad spores Fremgang i <b>Dannelse</b> og gode Seder , og navnlig fandt jeg , at de gamle Drikke-Vaner dog holde paa at tabe sig . Det staar ikke saa ilde til nu , som det har gjort ; jeg fik et bestemt Indtryk heraf i Almindelighed og mindes ogsaa enkelte Trcek , som tyde derpaa . En <b>Kone</b> fortalte mig saaledes ial Enfoldighed, at i hendes Ungdom var det saa vist , at der selv i Huse med ganste maadelig Formues- Forfatning blev brygget mindst en Tsnde Malt til Julen , men nu er det ikke saa aldeles almindeligt. — Men ligesom man kan undre sig over , at de gamle Drikke-Skikke have holdt sig saa lcenge her i Fjeldbygderne , saa fan man jo altid frygte for , at den gode Forandring ogsaa stal gaa langsomt for sig . Maatte denne Frygt blive bestjemmet! maatte der ret snart sverges den Tidende , at det livlige og kraftfulde Fjeldfolk har lert en bedre Brug for sine Evner , hvorved det visselig kunde udmerke sig skjsnt og herligt ! la , maatte dette ste snart ; thi hvert Aar , som gaar hen paa den gamle <b>Vis</b> , er farligt !

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012070608268'target=_>Sundt, Eilert, 1867, Husfliden i Norge</a><h4>1535</h4>Ord og udviste Vandel . Dcn Travclhcd , som <b>Kvinden</b> stcdsc har befnndct sig i , har vist ogsaa vcerct til stort Gavn baade for at vcrligeholde Velstand og have hende i loindclig HnZligbed og Tyd som god Borneopdrager , hvilket jo er kemmct den hele Slagt indcn Bvgdcn tilgode . Men det maa ogsaa virtelig crljcndcs , at hun i denne Tid ogsaa har det altfor travelt , da hun ved de mange Arbeider hindres baadc i legemlig og <b>aandclig</b> <b>Uddannclse</b> . I de sidste Aar har man faact en Fristolc i Gang for de konsirmercde Drengc inden Gjcesdals Sogn , som holdcs i 4 6 Uger aarlig . Men for <b>Piger</b> har man endnn intet Saadant gjort og kan heller ilke tanke paa nogct , da de slet ilke har Tid . Tet cr saaledcs lun faa <b>Piger</b> , der komme ndcnfor sine egne Tore for at låre nogct , og det cncste , som disse faa låre , cr Syning af Kladcr , dog med ct Tilfalde undtagcn, som var , at cn <b>Pige</b> var i Merilcrre z Aar , hvilken Tannelse bun ogsaa senere har benyttet til Nytte for sig og sit At <b>Kvinden</b> ogsaa hindres i lcgcmlig <b>Uddannelse</b> , maa antages deraf , at gamle Foll siger her ere sicre sysscliqe Piqer nn end for . Vel cr dct jnst ilke dcn almindclige Mening , at dette siulde have sin Grund i Spindingen cller a ! ldct Arbeide, men dct ansccs hcllcr som cn Folge as Modesygcn og Bcklcrdningcn m . m . Og stjont dctte for cn Dcl ilkc cr faa ugrundct , saa tror jcg dog at maatte tillcrgge Spindingen ligc saa stor medvirkcnde , om isse stsrre Aarsag , eftcrscm dcn er saa vedholdcndc fra dcn tidlige Morgcn til dcn sildige Aften i den storste Del

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100803009'target=_>Becker, Karl Frederik, 1851, Karl Frederik Becker's Verdenshistorie</a><h4>377</h4>De grcrste Hetcerer ere meget omtalte . Det blodagtige Lille-Asien , i Scerdeleshed Milet , var disse Personers Fcedreneland og Plantestole ; de fleste og rigeste fandies i Korinth , hvor de endog node et Slags Ndmcrrkelse ^ de beromteste i Athenen . Allerede Solon , der ved strenge Straffelove imod Wgtestabsbrud og F ^ rforelse havde sogt at sikkrc Borgerindernes Sceder , taalte Hetcererne og tilstod dem Beffyttelse , og , hvad et Fragment af en komisk Digler bevidner , i samme Hensigt , thi i Folge antikc Begreber kom fremfor Alt Borgerne og deres Familier i Betragtning , men Hetcererne vare Fremmede eller Ufrie , hvis moralste Tilværelse under alle Omstccndigheder kunde opoffres til Bcdfte for hine . De sogte , i Scerdeleshed i Athenen , ikke blot at glimre ved Skionhed , men ogsaa ved <b>Aand</b> og Charukteer . Det gielder iscrr om A spa si a , den heromteste as alle Hetcercr , hvem Perikles , efterat hun lcrnge havde vceret hans Veninde , tilstdst endog crgtede , og som bavde en stor Indstydelse paa ham , af hvilken Grund begge derfor ikke heller undgik Komedieskrivernes bittre Epot . Hvor udmcerket hendes <b>Aand</b> og hendes <b>Dannelse</b> maa have vcrret , seer man af Plato , der i Menennus lader Sokrates sige , at hun har vceret hans <b>Lcererinde</b> i Veltalenhet», hun , som havde dannet mange andre store Talere og ogsaa den storste af dem Alle , Perikles . Men Aspasia gjorde ogsaa en Undtagelse fra de ovrige <b>Fruentimmer</b> , til hvis Classt hun blev regnet .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008112403010'target=_>Büchsel, Carl, 1862, Erindringer af en Landsbypræsts Liv</a><h4>338</h4>Den fattige <b>Pige</b> og den fattige Karl , der maa syge at staffe sig Brydet med at tjene , ere Byrn af fattige Forceldre , der have maattet arbeide i deres Anfigts Sved for at staffe dem Fyde og Klceder , og man skulde synes , at man kun behyvede at vcere i Besiddelse af en lille Smule menneskelig Medfølelse for ikke at behandle dem som blotte Arbeidsmaskiner og troe at have opfyldt sin Pligt imod dem ved at udbetale dem deres Lyn , men saavidt muligt <b>vise</b> dem en Faders og Moders Kjcerlighed . Naar fyrst Tienestetyendet igjen faaer lcert at cere og elste Herskabet , saaledes som Guds Bud krcever det af dem , saa ville ogsaa Dagleierne lidt efter lidt lcere at cere deres jordiske Herrer og at frygte Herrernes Herre . De Veie , der fyre til dette <b>Maal</b> , ere meget besvcerlige , men derfor maa man ikke blive treet . Vildfarelferne og Overtrcrdclserne ere svcere og store , og derfor maa ogsaa Omvendelsen vcere alvorlig og grundig , og naar man ikke kan faae udrettet noget Stort , saa maa man nyies med det Mindre .

<h4>381</h4>at de stulle vcere Forceldrene lydige , om en Prcest hedder det scerligt , at han bor have lydige Born . Der er ikte oftstillct den Fordring , at en Prcest stal have omvendte og troende Born , men kun , at han stal have lydige Bsrn : det er vistnok en sygelig Pietismes Vildfarelse , naar man vil paatvingc Byrnene Former og Ord , som give dem Gudfrygtighedens Skin ; i Reglen ville de lade det blive ved Skinnet , og destomere give de da Anledning til Forargelse af enhver Slags . Man taler hel foragteligt om Pensions- og Oftdragelsesanstalter, i hvilke man bibringer unge <b>Piger</b> en <b>vis</b> Tournure og Dressur ; men at dressere Nogen til Fromhed avler kun Hykleri , der er meget vcerre end et verdsligt Vcesen og Liv . Al Opdragelsen byr egentlig vcere Bortdragelse fra Synden og Hendragelse til Christus , og den bedste <b>Dannelse</b> er den , som gaacr ud paa at spre det tabte Gnds Billede tilbage . Derfor kan en Tjenestekart og en Tjenestepige godt have en sin <b>Dannelse</b> og vcere velopdragne . Al anden Oftdragelse, som sigter til et andet <b>Maal</b> , sprer enten til Selvforgudelse eller til Mennestefrygtens Trceldom . Naar Johannes den Dyber siger :  mig byr det at aftage , ham at voxe , " da har han dermed udtalt Principet for al Opdragelse. Faderens og Moderens Indflydelse byr aftage , men Kjydet og det gamle Menneske byr ikke voxe , men Han , Herren byr voxe . Frygten for den levende Gnd og Kjcerligheden til Herren er det , som al fornuftig Opdragelse byr pleie i Byrnenes Hjerter , og det <b>Maal</b> , man maa syge at naae , for at de kunne vide , at Guds Vje seer ogsaa der , hvor Faderens Aje ikke seer dem , og at Herren er dem ncer , hvor Faderens Haand ikke kan beskytte og lede dem . Den Opdragelse , som kun har Barnets fremtidige borgerlige Stilling for Nje , og som kun gaaer ud paa at gjyre det dygtigt til at erhverve sit Vryd , fyrer til de vcerste Resultater ; thi Kundstab og Dygtighed opblceser og giver iugeu Beskyttelse imod Fristelserne og ingen Kraft til at stride imod det gamle Menneske . Fremfor alt byr Forceldrenes Byn lede og bcere Barnet og bygge Boliger til det . Moderen underviser de

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016050948142'target=_>Riddervold, Jul., 1878, Pauli Brev til de Efesier</a><h4>663</h4>Glad og tillidsfuld overgiver Apostelen alle sine Begjæringer til ham , som har sin Gloede i at hore sine Borns Bpnner , og som efter sin Naades og Magts Rigdom formaar at gjore langt over vor Begjæring . Det hedder eg. : som kan gjore udover Alt , mere end Alt , det er , som har den hoiefte Magtfylde . — „ Overflødigere end vi forståa . " Dermed henstiller Paulus til Guds <b>Visdom</b> at opfylde hans Bon . Thi , „ vi vide ikke , hvad vi skulle bede , som det sig bor " , derfor stal vi lade Gud raade , eftersom hans Tanter er hoiere end vore , og han alene ved , hvad der kan fremme vor evige Frelse , og det rette Middel og Vei dertil . I sine Belj endelser fortoeller Augustin , at hans <b>Moder</b> Monika bad , at Herren vilde forhindre hans Reise til Rom , den forforende Stad , hvor hun frygtede , at hendes Son vilde <b>aandelig</b> aldeles gaa tilgrunde , men Herren forhindrede ikke hans Reise og bonhorte dog Moderens Bon ; thi hendes bestandige Bon var Sonnens Omvendelse , og den stede netop paa denne Reise . Augustin siger : Du gav hende ikke , hvad hun dengang bad om , for at du kunde gjore med mig , hvad hun stedse bad om . — Efter den Magt , fom ter fig kraftig i os , nemlig i Troms og Gjenfodelsens Naadevirkninger i Hjertet . Den Gud , der har aabenbaret sin Naades Magt i sine Troende ved at

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012022908088'target=_>Borchsenius, Otto, 1880, Literære feuilletoner</a><h4>88</h4>Hvad Fru Brun derimod angik , siger Oehlenschlager , gjorde hun med sin elskværdige Personlighed , sin sjeldne <b>Aand</b> og for en <b>Kvinde</b> sjeldne <b>Kundskaber</b> Erobringer , hvor hun kom . fra Fyrsteborgen ned til Hyrdehytten , og der var næsten ingen Mand eller <b>Kvinde</b> paa den Tid af nogen Berømthed i Danmark , Tydskland , Schweiz og Italien , som hun ikke kjendte , ikke havde staaet i venskabelig Forbindelse med , hvis Yæsen hun med Fantasi og Forstand ikke , mere eller mindre , havde opfattet og kunde give Træk og Billeder af . Dette bidrog meget til at gjøre hendes Omgang behagelig ; man hørte hende gjerne fortælle , og da hendes Døvhed tog til , saa maatte hun ogsaa blive mere interessant i Tale end i Samtale . Hvad der foregik omkring hende , kunde hun ingen ret Del tåge i , og denne hendes Døvhed , i Forbindelse med den Magelighed og Uafhængighed , som Eigdom forskaffer , gjorde hende efterhaanden mere til en deltagende Tilskuerinde at Livet , end til en hjertegreben , selvhandlende Person .

<h4>2291</h4>Almqvist optræder allerede som Forfatter i 1814 , men kommer dog først en Snes Aar senere til at spille en mere fremtrædende literær Rolle . ludgangen til hans Liv , hans Deltagelse i Manhemsforbundet ( se herom bl . a . St . » Det 19. Aarh.c . Januarheftet 1876 ) , er noget drøj at komme igjennem . Fantasteriet grænser saa nær op til Forskruethed og Vanvid , som paa nogen Maade muligt . Des større Interesse har Flytningen til Vermland, hvor Almqvist i to Aar ( 1824 - 26 ) paa Rousseansk <b>Vis</b> søgte at leve som s > idealiseret Bonde « . Geijers Advarselsord hjalp ikke . » Den borgerlige Sfære « , skriver Geijer , » hvortil man er født og opdraget . danner omkring Enhver et Slags naturlig Atmosfære . Faa gaa udenfor den aden at fcabe Yejret . Yed en lidt efter lidt erhvervet <b>Dannelse</b> kan man stige op ad . Ulykkens ihærdige Skole kan ogsaa lære En at udholde Faldet nedad . Frivillige Spring i den ene eller anden Retning lykkes sjeldent . « Almqvist skulde sande disse Geijers Ord , men dog først efter at have drevet sin Plan saa konsekvent igjennem , at han endogsaa havde ægtet en Bondepige , hvilket fik en skjæbnesvanger Indflydelse paa hans senere Liv . Ahnfelt paapeger saaledes træffende , hvilken Sammenliæng der tinder Sted mellem Almqvists Liv og Digtning . Han havde troet at kunne realisert : ' sine Drømme om et idyllisk Liv i Naturens Skjød med denne sin Elskede , men da han senere trængte til en <b>Hustru</b> , der kunde folge ham paa hans Tankers Flugt . brast Drømmen , og da han rundt omkring sig saa det ene ulykkelige eller uharmoniske

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112301028'target=_>Keyser, R., 1866, Norges Historie</a><h4>3295</h4>saa underlagde Haakon sig paany alle Derne , gjorde siden for at udsone sin Brede en Pilegrimsreise til Rom og Jerusalem og blev ester sin Hjemkomst meget elstet af Orknoingerne for sin kraftfulde Landsstyrelse og sine gode Love Han efterlod ved sin Dsd larledommet til sine to Ssnner, Paal og Harald ; den sidste havde Haakon avlet med sin Frille Helga , en Datter af Mod dan , en fornem Mand paa Katanes . Paal Jarl var fredelig og venscrl ; Harald blev meget ledet af sin <b>Moder</b> Helga og hendes Soster Frakork , der begge holdtes for lumske og tryllekyndige. Brsdrene havde allerede fra Barndommen af vceret uenige , og da Harald lod drcebe Paals Ven Thorte l Fostre , kom det til aabenbari Fiendskab mellem dem . Kort efter dsde imidlertid Harald Jarl pludseligen , som det hed , drcebt ved en fortryllet Kjortel , som af Helga og Fraksrk var virket og bestemt for hans Broder , men som Harald mod deres Villie ifsrte sig . Paal drev nu Ssstrene bort fra Derne og var en Stund ene Jarl 2 ) . Men Magnus den helliges Ssster Gunnhild var af Kong Magnus Barfod bleven giftet med Kol fra Agder , og deres Ssn . den fsrncrvnte Ka le , troede sig at have Ret til sin Morbroder Magnus ' s Dccl af Orknoerne . Kale var i Skjsnhed, Raffhed og <b>Kundskaber</b> en af de mest udmerkede Ynglinger i Norge . Han kunde — som han selv ien <b>Vise</b> siger — ni Idrcetter ; han var svet i Bretspil , Runer , Lcesning og Smedekunst ; han kunde wbe paa Ski , flyde med Bue og ro ; han forstod at siaa Harpen og var Skald .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009082403014'target=_>Ewald, Johannes, 1853, Johannes Ewalds Samtlige Skrifter</a><h4>698</h4>og det er vist , at han bedre kunde gisre Regnskab for hvorledes en rommcrsk <b>Dame</b> , end hvorledes hans egen <b>Kone</b> saae ud i sin Pynt . Men det , som er det merkeligste , er at han baade kiendte og elskte , ja folle det Sisnne i en Homer , en Sophoclcs , en Pindarus , en Horats , og dog tillige var saa afsagt en Fiende , saa fuldkommen en Foragtere af al nyere Poesie, at man kunde jage ham paa Flugten med at <b>vise</b> ham noget, der legnede et hollandsk Vers . Naar jeg lcrgger til , at Herr Oldsoker ( saaledes vil jeg kalde Rectoren ) i sine Scrder , som man ventelig kan formode det , var bizar , paastaaende , gnavvurn og overalt temmelig pedantist , saa kiender man ham saa noie som man behover at kiende ham . Den ham tilfoiede Conrector , som afvexlcde med ham i Disciplernes Underviisning , var noget ncrr hans fuldkomne Contrast , og alt det hvad Herr Oldsoker ikke var ; en Mand , fuldkommcn skabt og dannet for den Tidsalder , vi leve i , just ingen Foragtere af det Gamle , men en langt stsrrc Elskere og Kiendere af det Nye , ret graadig ester hollandske Vers , og ydermere selv en Poet ; desuden en Mand af Levemaade , munter , sagtmodig , beleven , og ikke mindre end Pedant . Man maa i mine Tanker paa ecngang undres over hvorledes disse to Mcrnd saa ncrr paa Siden af hinanden kunde forliges , og formode , at deres naturligviiis forskiellige Levemaadc maatte forvilde den storste , og verre til sand Fordeel for den kloge Deel af Diseivlerne . Dette havde og rigtig nok Sted , og dersom Herr Frankhuysen nogentid havde en Fordeel i Klogffab og Lykke fremfor Herr Schliker , saa bestod den deri , at hiint anvendte begges <b>Kundskab</b> paa at udvidc sin egen , uden stort at bryde sig om deres Fordomme ; da derimod denne , som desudea i Forbigaaende sagt havdc et langt svagere Nemme , saa lcrnge vexelviis hveranden Dag afskycde det Gamle og det Nye , at han tilsidst i Hiertet virkelig afskyede

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092303045'target=_>Lassen, Albert, 1834, Lærebog i den almindelige Historie</a><h4>1178</h4>Den reformeerte Troslere , som den kaldes i Modsetning af den evangelist-lulherste , fik dog forst stn fulde <b>Uddannelse</b> ved Cal vin . Denne Reformator , Johan Chauv in var hans egent , lige og Calvinus derimod hans lerde Navn , var fsdt i Noyons , / > g opdroges af en relstaf , fen , men overordentlig streng Fader og en from <b>Moder</b> ; disse Foroeldrenes Egenskaber indpregedes tidlig i den ungdommelige <b>Sjel</b> . Han gik siden i Skole i Paris , udmerkede sig fremfor alle sine Meddisciple , og solte sin aandelige Overlegenhet » , en Fslelse , som han siden ikte tnnde eller vilde undertrykte , og som endog git over til Haardxakl kenhed . Han siuderede forst Theologie og blev Prest , men ved Bibelens Studium og desuden ei « bekjendt med de nye Neformationers Grundsetninger,

<h4>1856</h4>For sjette Gang formelede Kongen sig 1643 ; hans Valg traf da Catharina Parr , Enke efter en Lord Latimer . Da hun var Neformationen yderst hengiven , saa vandt Cranmer i hende en meglig Stolte imod Gardiner og Katholikerne , og freme mede Neformationen ved alle Midler som Kloge stab tillod ham ; han havde ei lagt Dolgsmaal for Kongen paa sine Tanker om hans sex Artikler ; men ikke desiomindre beholdt han dog hans Gunst . En Plan til at styrte saavel ham som Dronningen mislytkedes , og endtes med Biskop Gardiners fuldkomne Unaade . Denne Prelat havde nemlig bygget sin Plan til Dronningens Fald paa Kongens Intolerante og det mistroiske Sind og vrantne Lune som beherskede ham . He » rik var i si » e sidste Leveaar meget sygelig , og hans Legeme opnaaede en uhyre , meget besverlig Fedme . Hans Mistene kelighed , Vrantenhed , Serhed og opfarende Vrede gjorde ham saa vanskelig at omgaaes , at Ingen nennede sig ham uden med Frygt . Neligionse samtaler vare fremdeles hans Kjephest , thi de gave ham Anledning til at <b>vise</b> hans Skarpsine dighed , hvoraf han gjorde sig saameget til . Haus Gemalinde , som var en <b>Dame</b> af megen <b>Aand</b> og <b>Dannelse</b> og som havde megen Indfiydelse over

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008092603002'target=_>, 1837, Samling af Love, Tractater, Kundgjørelser, Resolutioner, Departementsskrivelser m.m., som siden 1814 ere udkomne</a><h4>3815</h4>fri for Snee , og vedkommende Eier eller Bruger , samt den , der efter Amtmandens Foranstaltning <b>moder</b> paa det Offentliges Vegne , gives 8 Dages Varsel . Ved Taxiens Bestemmelse bor Jordens storre eller mindre Frugtbarhed , fordeclagtige eller siette Beliggenhet » og dens Vcerdie i Forhold til Jorder af lignende Storrelse og Beskaffenhet » i samme Egn , tages i noieste Betragtning. Naar Jord i forommeldte Diemed afgives af Statens eller benesiceret Gods , tilfalder Godtgjorelsen i fsrste Tilfoelde Statscassen , i sidste det beneficerede Godses Fond , hvorimod davcerende Opsidder nyder saa stort Afstag i de aarlige Afgifter som Renter af Erstatningssummen belobe , beregnet til 4 pCt . ^ ) Anlegges ny Vei over Jord , som privat Mand har bortleiet , tilfalder Erstatningen Grundeieren , som derimod er pligtig at give Leieren , for dennes Besiddelsestid , en aarlig Godtgjsrelse , der bestemmes til 4 pCt . af Erstatningssummen . 26 ) Den gamle Vei stal , naar den tilfalder den , der maa afgive Jord til en ny , beregnes efter Arealindhold og Jordens Beffassenhed . 27 ) Hvis Gjerder , formedelst Veies Anlcrg eller Forandring , maae opfsres , borttages eller flyttes , fial Erstatning for det Tab eller de Omkostninger , saadant medforer , sørskilt beregnes og godtgjores Eieren . 28 ) I Skov og udyrket Mark maa Steen , Gruus , Jord og Sand til Veies Anloeg eller Istandsettelse tages , hvor saadant ncermest Veien er at faae , og uden Skade for Eieren eller Brugeren kan stee , men derimod ikke fra indhegnede og dyrkede Marker , forinden Godtgjorelse derfor er bestemt ved lovlig Taxt , saafremt mindelig Forening ei derom kan opnaaes . 29 ) Den Godtgjorelse , som efter §§ 25 , 27 og 28 udbetales Vedkommende , tillegges de svrige Veiomkostl ninger , og udredes efter de samme Regler , som ihenseende til disse ere fastsatte < " § 2 ) . Tarationsforretningerne afholdes paa Veivesenets Bekostning . 30 ) Erstatning for Aftredelse af Jord til Bygdeveie , paaligger det vedkommende Bygd at udrede . 31 ) Ret til Erstatning for Skade , som maatte vere Nogens Grund tilfsiet i Anledning af Veiarbeide , bortfalder , naar Vedkommende ikke fordrer Erstatningen inden 1 Aar efterat Skaden er steet . 32 ) Ingen maa forhindre Veibestyrelsen fra at give det Vand , der er til Skade for Veien , fornsdent Astsb . 33 ) Ved Hovedveiene bor paa hver H Miil oprettes Milepels , hvilke gives forskjellig <b>Dannelse</b> , eftersom de <b>vise</b> enten hele eller halve Mile . I ethvert af Rigets 4 Stifter bor Heel-Milepcelene med tydelige Tal <b>vise</b> Afstanden fra Stiftsstaden . 34 ) Hvor to eller flere Hovedveis stode sammen , bor opsettes Veivisere med tydelig Paategmng om , hvorhen enhver Vei forer . 35 ) De med Milepceles og Veiviseres Anskaffelse og Opsettelse forbundne Omkostninger , udredes ved Ligning paa ethvert Amts Matriculstyld . 36 ) Til at betegne Delet mellem Ro') Jfr . Res . 20 Juli 1832.

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008110603037'target=_>, 1852, Barlaam og Josaphat</a><h4>412</h4>for det , som er modtaget , om det end er lidet . Men jeg er , som du ser , en faltig og gammel Mands Dalter, og jeg takker med Glæde Gud for hans Gave , og jeg ved for vist , at nåar det behager ham , kan han give langt Storre og Mere . - Men Alt , hvad der henhorer til denne Verdens Pragt , hvilket vi have i Forvaring og tilsyneladende eie , bliver ofte til Gavn , men oftere til Skade , og forgaar dog omsider altid paa samme Maade . Men Gud har givet mig langt nyttigere Gaver , for hvilke jeg stedse bor takke ham ; thi ved hans Gave og Miskundhed er jeg skabt efler hans Billede og har faaet <b>Kundskab</b> lii at kjende ham selv og faaet Fornuft fremfor andre Kreature , og ved hans Naade Indbydelse til evig Salighed , og har lært at forståa hans Hemmeligheder . Den himmelske Herligheds Dor er ogsaa uden Vanskelighed aaben og beredt , dersom jeg selv vil gjore mig den Umag al stræbe dertil . For saadanne Tings Skyld og for de hans Gaver , hvilke ligesaa vel Fattige som Rige modtage, bor jeg altid prise min Herre ; men dersom jeg ikke gjor det , ved jeg ikke , hvilken Undskyldning jeg skal forsvare mig med. " Da den unge Mand horte disse <b>vise</b> med Veltalenhed foredragne Ord af Pigen , kaldte han hendes Fader til sig og tiltalte ham saaledes : „ Jeg finder din Datters Ord saa <b>vise</b> , at jeg anholder om hendes Kjærlighed , og jeg beder dig , at du vil lade mig ægte hende med Ære . Slige Ord eller Udtryk har jeg ikke hort for af nogen <b>Piges</b> Mund . " Men hendes Fader , som kjendte den unge Mands Herkomst og Rigdorn , svarede ham saaledes: „ Det sommer sig ikke for dig , som er oprunden af saa hoi Æt og saa rige Folk , at ægte en

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010021803053'target=_>Vig, Ole, 1857, Norges historie indtil Harald Haarfager</a><h4>1583</h4>Da nu Kvceldsmaden kom ind , gav Erik sig til at bide af alle Stykkerne , kastede dem fra sig og spurte , om Sligt var at byde fremmede Folk . Tjenerne blev skamfulde og bar fram saamcget som til et stort gjildt Gjcrstebud . Imidlertid fik Kongen hore mangt et trcrffende Ordsprog ;  Kunstab vokser ved- <b>Lcerdom</b> og Undervisning ; den <b>Vise</b> stal lcere af den Viseres sagde Erik ; men saa forlangte han Drik til Maden . Frode syntes , det var en uforskammet Gjcest , men Erik blev ikke Svar skyldig .  Faa kjender Tiendes Tarv , " sagde han ; og Kongen bad da sin Sester Gunvor komme ind og stjcenke ham . Da hun nu bragte ham Drikkehornet , greb han ogsaa . Hacmden med det samme og spurte :  Vil du . Konge , give miss ^ hvad jeg holder her ? " Frode tccnkte kun paa Drikkekaret og sagde la , hvorpaa Erik trak Prinsessen til sig .  Nu ser jeg . da tydelig , " sagde Kongen ,  at du er baade grov og dum , for borti llands plejer vi at lade <b>Pigerne</b> have sin Frihed . " lasaa," svarcde Erik ,  har jeg tagct mere end jeg fik , saa kan det blive omgjort ; men faar jeg ikke det Hele , vil jeg dog beholde min Del , " og gjorde saa Mine til at ville hugge Haanden > af Prinsessen . Kongen sagde da , at han gjcrrne for hans Skyld maatte faa Gunvor .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006081700019'target=_>Pavels, Claus, 1864, Claus Pavels's Biografi og Dagbøger</a><h4>2390</h4>Bugge er ncesten fortvivlct . Han siger at dette Storthing fuldender den Skjcendsel , som sidste Aars Felttog paadrog Norge . Han kalder det baade en Ulykke og en Vancere for enhver retstafsen Mand at voere valgt til Folkereproesentant , bruger de haardeste Udladelser om Christie, og anscr det for afgjort , at vor Selvstendighet » som Nation om faa Aar gaar forloren . Han tiltror ikke Regjeringen, allermindst Kronprindsen Andel i det som ster ; derimod mener han , at de Svenske med hemmelig Fornsielse agte paa Vegivenhedernes Gang , da de se , at vi , ved at raade os selv , allervissest ile det <b>Maal</b> imsde , hvortil be onske at lede os , og at vi , for at undgaa Anarchi og Tilintetgjørelse maa kaste os i deres Arme . Skylden til den lumpne <b>Aand</b> , der styrer vor Foerd sinder han i vor fordums Forbindelse med Danmark og det religiose Fritcenkeri, vi derfra har hentet . Gud veed , hvorfra dette bitre , blodige Had til de Danfke kommer . Han har , ligesom jeg , boet lcenge iblandt dem , har som jeg havt to Prcestcembeder der , hans <b>Kone</b> er endog en Danmarks

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014061108129'target=_>Thiers, Adolphe, 1847, Consulatets og Keiserdømmets Historie</a><h4>597</h4>I Malerkonsten var en paa Grund af Hensigten agtvcerdig Reaction indtraadt mod det attende Aarhundredes Smag . I dette blodagtige og philosophifle Aarhundrede havde den under Regentskabet tilbedte Maler Voucher med dristig Haand fremstillet letfcerdige Courtisaner , som ikke fortiente Opmærksomhed paa Grund af deres Skionhed , men fordi de vare i Besiddelse af en <b>vis</b> vellystig Inde . Den paa en cedlere Maade begeistrede Greuze havde i Modsætning hertil med fiin og saftig Pensel malet fortryllende unge <b>Piger</b> . Men den af Voucher fornedrede Konst var ikke bleven hcevet af Greuze til den Vcerdighed , som Poussin , stisnt han manglede Geni , vidste at forskaffe den . Det forundes kun en enkelt Gang en Nation at <b>vise</b> Verden Michel-Angelos og Raphaels Geni , men enhver Nation , som dyrker Konsten , bsr strcebe efter en reen og cedel Stiil , et <b>Maal</b> der kan naaes ved grundige Studier . Dette <b>Maal</b> havde den beremte Maler David naaet . Han foragtede det Prceg , som Konsten havde faaet i hans Ungdom ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2007081003009'target=_>Daa, Ludvig Kr., 1841, Svensk-norsk Haand-Ordbog</a><h4>3926</h4>BMemzi , s . pl . n . <b>Maal</b> , Djemed , Hsnsigt . Syl , - ar , rø . Syl . Syllik , aåj . syldannet. Sylmakare , s . pl . n . Smed , som gjor Syler . B ^ lBlHn « jL , - or , f . et ( Vlags Insekt : / t » / i ^ ick ' « . Sylviiska, - or , / ' . Sylflltteral . Sylort , - er , / . ( Plante ) Eylblad , Frytebue 9 ? . : Subularia . Syll , - ar , , n . Njcelke , som lcegges til © runb , © oil , Syl N . Svllogism , - er , rø . syllogisme . Sjlt , « . Syltetsj . Sylta , v. a . i . Sy . in , sylte , nedsylte , lcegge ned , drf . B ^ ltn ! nF , f . Sylta , " / . Svlte . . Syltmangelska , - or , / . <b>Kone</b> . som scrlger Sylte . Syltl,ufvu6) ii . et i Sylte lagt Svinehoved. Syltburk , - ar , rø . Spltekrukke . Syltkål , n . nedsyltet © uurfaaf . Syltrajolk , rø , sur gaaremelP , Syltenielf. Syltsaker , pl . Syltetoj . Sylfskal , - ar , rø . Syltekrukke . Syn , - er , rø . Syn ( i alle Ordets Vetydninger). 2 ) ( dagl . T . ) Anssgt . Aasyn. 3 ) Synsforretning , Vesigtelsesforretning; midt i synen , midt i Sjne : ne ; det bar . syn for saga el . 88. FN , bet er Syn for Sagen ; for syns Bli.ull , for et Syns skyld , paa Skromt ; halla . syn , afholde Synsforretning ; vara el . se i syne , have Syner ; Rlll ^ r ^ n , en Synsforretning , hvor Statens fornemste Embedsmcend ere Sunsmcend , og fra hvis Dom ej kan appelleres , Overskjon . Syna , v. a . 1. syne , bessgtige , drf . Synande, n . Syning , f . Synagoga , - or , / . Snnagog , lodskirke . Synas , v. d . 2. 2. ( I . synast ) synes ; sees , lade ffg tilsyne , <b>vise</b> sig . B ^ naxel , - xlar , rø . Svnakse / Ijeakse . Synbar , adj . Synbarligen , adv. synlig , ojensynlig , aabenbar , iojnefaldende ,

<h4>4904</h4>VirNa , M . fi . s . Hvirfla , M . fl. Virka , v. a . I . » irfe , » æee paa en kunstig Maade , f . E . en virkad Koll . Virkare , s . pli m . Virkerska , - or , / . Virker , drf . Virkning , f . Virkcri, n . Vlplln3l , - ar , m . Virke , n . Tråvirke , n . Vygningstr ^ - tommer , LEmne , ZEmninqstriV . <b>Vis</b> , * . / , i . n . <b>Vis</b> . Maade , Skik , Vrug ; på satt nek <b>vis</b> , paa en Maade . <b>Vis</b> , adj . vist , visligen , adv. <b>vis</b> . Visdom, m . Vishet , / . : <b>Visdom</b> . Vis- Ilet ^ lai-H , f . Visdomslære . Visnet»-tand, - tander , / . Visdomstand . Visa , v. a . \ . & 2. 2. <b>vise</b> , brf . Visande, Vi » nin ^ , / . Visen , Visning . Visa , - or , / <b>Vise</b> . Visbok , - bocker , / ° . Visebog . Visgubbe , - ar , 7 « . Vis- Kumma , - or , J . Mand , <b>Kone</b> som fcefger Viser . Vismakare , s . fl. m . Nisemager . Viston , - er , m . Visetone. Visare , s . pl . m . Viser . Visartafla , - or , jf. Ur ! ? ire . Vi » « rverk , s . pl . n . de forreste Hjul , som sidde mellem Uret selv og Skiven . Visbok , m . fl. f . Visa . <b>Vise</b> , - ar , m . Ploglob , - hoved ( hvori Skj < eret befcrstes ) . 2 ) s Simober , Bidronning, Viser . Visebo , - n , « . Dronningselle . Visefall , s - pi . « . Mangel paa Dronning . " Visclos ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014052208033'target=_>Marsh, Catherine M., 1880, Kapitain Hedley Vicars's Liv og Død</a><h4>132</h4>tidligere Aar ; men maaste er dette Anste urigtig , da Alting er ordnet viseligen for os i Forsynets evige Naadslutning . Jeg tror ikke , der findes Nogen i hele Regimentet , der var funket dybere ned i Sund og Uretfærdighed, end den , som nu striver dette om sig selv . Jeg var ganske en Syndens Slave , da jeg ankom til Halifax . Det var Deres Prædikener , som ved den Helligaands Naade virkede med frelsende Kraft vaa mit Hjerte , der standsede mig i SyndelMet , og jeg haaber , at jeg nu omsider er bragt fra Mærket til Lyset og fra Uvidenhed om Gud til <b>Kundskab</b> om Hans Veie . Jeg anser mig selv som et levende Vidne om Guds Godhed , idet Han forundte mig Tid til Omvendelse og gav mig en Lwrer og Raadgiver , som ilte var bange for at bekjende Christus korsfcrstet og samvittighedsfuldt vrcedike Evangeliets store Sandheder . Jeg takker Dem af mit ganske Hjerte , dyrebare vr . Twinning , for alle gode Raad og Paamindelfer , som De stedse saa venligen har givet mig . Jeg kan blot <b>vise</b> mig taknemmelig vaa en Maade , og jeg ved , De vil fwtte Pris vaa den . Jeg har i flere Maaneder ikke bolet mit Kncr for Naadens Throne , uden at erindre Dem ; heller ikke stal De nogensinde mangle mine svage ForbMner . Jeg haaber , at De endnu maa blive Midlet til mange Sjceles Omvendelse . Vedbliv , som De hidtil har gjort , ikte at vide nuget Andet end Jesus Christus ag Ham korsfæstet , og da stal det lyttes Dem . Ingen anden Prcediken nytter eller stjcenker en eneste Trustens Straale til den , der kjender sig selv som en arm , fortabt Synder . Jeg har ikke Tid til at skrive mere . Vi ere nu indkvarterede i Barakkerne her i W . , men skal i Begyndelsen af nceste Uge flyttes til Tomer i London , og derfra i Juli til Leiren vaa Cobham Plads . Ved Mønstringen roste Generaladjutanten meget Regimentets Holdning. Toml , jeg har endnu ikke vceret istand til at erholde Permission for at beftge min <b>Moder</b> , uagtet jeg ncesten ini Aar ikke har set hende . Bring min kjærlige Hilsen til Deres <b>Hustru</b> . Jeg tror hver Officier i Regimentet statter hendes Opmærksomhed og moderlige

<h4>214</h4>Da jeg gik ud af Kirken mMe jeg Major H . , hvorpaa vi fulgtes hjem . Jeg havde en meget opbyggelig Samtale med ham . Christi Sag synes at ligge ham paa Hjerte , og han lamges meget efter at kjende mere af Frelseren gjennem Erfaringen . Han bad mig drikke The hos ham , og jeg saa da hans <b>Kone</b> for ftrste Gang , thi hun var syg , da jeg beftgte ham . Vi tilbragte en opbyggelig Aften sammen . Jeg gjFr meget af dem Vegge . For nogle Dage siden havde jeg en opbyggelig Samtale med en ung Korporal , som for kort Tid siden havde forladt mit Kompagni . I ncesten tre Aar har han vceret en trofast Christi Efterfølger , og jeg er saa dristig at sige , at han fremdeles „ strider Troens gode Strid . " Men vi maa ogsaa vcere forberedt paa at erfare det Bitre saavelsom det Ssde . Det bedr Mer mig at maatte fortcelle Dem , at En , fom jeg havde godt Haab om , er gaaet tilbage igjen stakkels Menneske , jeg beklager ham inderlige«! Jeg haaber , at det skal blive en advarende <b>Loerdom</b> for mig til at vaage og bede med stMre Vedholdenhed og Iver , og vogte mig , at ikke ogsaa jeg skal falde . Men jeg vil aldrig opgive Nogen Jesus opgav ikke mig , og jeg haaber og venter , at han skal blive bragt tilbage igjen paa Sandhedens Vei Jeg Mer , at det , De fagde i Deres Brev om <b>aandelig</b> Hoffcerdighed , undertiden retteligen kan anvendes paa mig , men naar jeg saaledes Mer mig tilbolelig til at forglemme hvo og hund jeg er , ftger jeg at forestille mig Synderen staaende alene , uden en Frelser og uden den Helligaand , og denne ulykkelige , sørgelige Tanke bringer mig straks til at ftle og erkjende min totale Fordcervelse og Elendighed . Formeld Alle min christelige Kjcerligheds Hilsen . Og nu : „ Vor Herre Jesus Christus og Gud vor Fader , som af sin Naade har elsket os og stjcenket os et fast Haab om evig Salighed , Han styrke Eders Hjerte og befceste Eder i al god Gjerning . Stedse Eders hoit forbundne og hengivne Ven . H . V . " I Begyndelsen af Februar Maaned modtog han Efterretning om , at hans Onkel , Oberst G . Vicars , var bleven farlig syg i Gibraltar . Paa Veien til Ostindien ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016070126004'target=_>Bache, Tollef Olsen, 1849, En Deel af T.O. Baches Levnetsløb</a><h4>297</h4>Vi have modtaget Din Skrivelse af Bde dennes , som Du er takket for ; vi see Dine Hjertes Fslelser , samt at de Grunde , vi handlede om , have faaet Indgang vaa Dit Hjerte . Betragt , Ssster , Christi Ord til hiin <b>Qvinde</b> , som af hans Ord blev saa oplivet , at hun priste den salig , som havde baaret eller diet ham ; men han svarede , at de vare salige , som Horte Guds Ord og bevarede det . Ikke bor vi agte Guds Naade ringe , naar han opvækker vort Sind og Hjerte , at vi fole den Kraft , som Ordet har ; thi naar hans Acmd faaer Rum hos os at forklare det , saa er det en Livets Lugt , som giver os fornyet Liv og Styrke til at lsbe Guds Vudords Veie . Men da maae vi vaa den anden Side tage os noie iagt , at ikke Fienden stal faae os i Sikkerhed, saa vi hvile i den stjcenkte Naade eller bygge vaa nogen Slags Gaver ; men soge at vandre fremad til det os foresatte <b>Maal</b> , lige stille , fort , at hvert et Trin førsigtig bliver giort , omgaaes i en ssnlig Frpgt og forarbeide vor egen Saliggjorelse med Frpgt og Bavelse ; thi uden dette steer , kan ikke vor Forstand oplyses , og vi komme til en grundig Kundstab om , hvad Herrens Villie er , saa de fijulte Ting i hans Ord blev os aabenbaret og forklaret ; thi da lærer Aanden os i Forening med Gudsord , hvad vi stulle gjore og lade , og vi liere ved daglig Erfaring at fijelne imellem det LEgte og Ucrgte , imellem Sandheds <b>Aand</b> og de antichristne Aander , saa om den kommer i en Lysets Engels Skikkelse og vil forfore os med egen udvalgt Ydmygheds Skikkelse , da antage vi ikke den , eller af Andre lade os forfore, men vi prove Aanderne , om de ere af Gud , og naar vort Vie er enfoldig , saa vi kjende os selv fattige og ringe , saa bliver Gudsord mere fatteligt og førstaaeligt for os , da fieer det , som Hauge har sagt , at Bibelen forklarer Bibelen.

<h4>524</h4>for mit eget Hjerte . Jeg deler aldeles Din Anskuelse deri , at det var godt at de , som ville reise om og bekjende Ordet for Andre , ei alene betragtede flittig Vibelen , saa de kunde holde fast ved Hovedet Christus , men og Franckes og fiere fordums Christne , saavelsom Hs . Liv og Vandel og fornemmelig hans Lcrre , saa kunde mange Vildfarelser rettes , som nu lidet agtes . Den Anmærkning Du gjsr : Ak ! naar vi de første troe Christne betragte o . s . v. , stemmer aldeles overeens med min Synsmaade . Gud give os Naade til at stride Troens gode Strid til vor sidste Ende , da vil hans <b>Aand</b> bevare os fra de antichristne Acmder , som indtrcenge sig i og iblandt os ; vi siulde da ikke troe enhver <b>Aand</b> , men prove dem , om de ere af Gud , hvilke Prsver maa fiee formedelst , hans Ord . Det mig tilsendte Coftie af A . Hs . Brev gjennemlceste jeg strar , men fandt i det en fremmed <b>Aand</b> , efter mit Kald . Jeg overleverede min <b>Kone</b> det , uden at gjore mindste Anmærkning, men hun stemte med det samme og fslte megen Bekymring derover . Siden har jeg selv flere Gange lcest og provet det ved den hellige Skrift , og finder det aldeles stridende derimod . Jeg havde allerede næsten strevet Brevet fcerdig til Dig , men den Tvivl , fom laae i mig , angaaende A . Hs . Brev , var mig meget folende , iscer da mange ikke selv ere saa noie bch ' endte i Bibelen , saa de ere blevne opvakte, have de dog ikke Magt til at prove Aanderne , og naar nu 2 Cor . 11 saavelsom 2 Thes . 2. 4 — 14 betragtes , saa sees deraf , at forskjellige Aander , som indtrcenge sig , kan ogsaa forfore deres Hjerter , som ikke vare onde . Vel indseer jeg , at en lceg Mand uden <b>Kundskaber</b> ei kan Mes i Ligning med den i 4 V . sidst anviste , men bliver Ordet forvendt, saa kan det udrette meget til Skade , da mange ere kaldte , men faa udvalgte . Betragte vi nu Match . 24,53 — 29 , saa sees deraf , at de falske Aander ere saa fine , at de Udvalgte ogsaa siulde forfores , om det var muligt ; men den , som holder fast ved Ordet , og tager dette til sin Regel og Rettesnor , den lærer at kjende Satans snedige Anlsb , ar

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008100603008'target=_>Becker, Karl Frederik, 1851, Karl Frederik Becker's Verdenshistorie</a><h4>386</h4>Alligevel lod en ved <b>Aand</b> , Veltalenhet » og <b>Dannelse</b> udmerket Mand sig itte afsirette fra at betrede denne Vei . Det var Tib . Sempronius Gracchus , som vi allerede have lert at tiende fra den nnmantinste Krig , en Son af den som Censor omtalte Gracchus , og en Dcrtterson af den eldre Scipio Africcmus . Hans <b>Moder</b> Cornelia, en <b>Qvinde</b> med en hoi <b>Aand</b> , der var dannet ved den greste Litteratur og Philosophie , havde afstaaet en egyptisk Konges Haand , for at kunne lede sine tvende Soimers Opdragelse .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016101748017'target=_>Wildenhahn, August, 1860, Martin Luther</a><h4>169</h4>ikke som Gud . Nogle sige , at Brodet og Vinen , samt Kjsdet og Blodet ere Eet ; Nogle sige atter , at det alene er Brod og Vin og intet Andet , end et Tegn , ligesom et Segl paa et Brev , ligesom vi ei skulde have Tillid til eller tro Gud uden Vrev og Segl ! " „ Af Eder kan man ci forlange nogen bedre <b>Kundskab</b> ! " svarede den ung / Mand igjen roligere- , „ I er saaledes undervist af eders Prcrstcr , at sammenblande Sandt og Falskt og at pcmbyrde Luther det Onde og Ucrrlige , som Evangeliets onde og ucrrlige Venner og Tilhcrngere gjøre . Dersom I ikke gjorde det saaledes , kom I heller ikke til Mcmlet . " „ I vil altsaa igjen ncrgtc det ? " fer Staldmesteren pacmy op. „ Har jeg ikke med mine egne Dine sect , at cd ' rs Folk , som udgive sig for gode Lutheraner, ci ville <b>vise</b> det hoicrrværdige Sakramente nogen ARre eller Mrbødighed ; men vende det Nyggen , staa og stirre lig et Trcrstykke og ville ikke bukke sig for Ham , for hvis Navn alle Kncr i Himlen og paa Jorden og under Jorden maa boie sig ! Nogle af Eder gaa til Alters uden al Anger og Bod , tage Sakramentet selv fat med deres syndige Hændcr , spise og drikke , ligesom om det var ganske almindelig Mad og Drikke , stikke Hostien i Lommen , bcere den hos sig , lege med den og — det er Kval at hore det for christclige Dren — kalder det en Afgud og Djævel og træder det under Foddcrnc . Vil I ncrgte dette ? Har jeg ikke selv værct tilstede , hvor Nogles Hoveder for denne Sags Skyld af Skarpretteren ere blevne afhugne ? " „ Herre , " for Smeden op , „ og det skulde den troende , fromme Martin Luther gjore ? Saaledes forestiller I Eder en crrlig evangelist Christen ? Er I tilfreds , om jeg kalder Paven , eders Herre og alle eders Papister Blodstjcrndcre og Mordere, fordi Pave Alexander den Sjette har været en Saadan ? " „ Det er heller ikke noget Under , om Saadant sker , " svarede Staldmesteren uden at agte det Ringeste paa de til ham rettede Sporgsmaal . „ De Lutherske have jo nylig i Nickclsburg i Mcrhren offentlig disputeret om , at Christus ikke er Gud : men blot et almindeligt Menneske og en almindelig Profet ; ja Nogle sige ligeud , at de ikke bryde sig Noget om ham , sige med uhort Forhaanclse af hans allerhelligste <b>Moder</b> , den nbesmittcde Jomfru Maria , at da han selv var undfangcn i Arvesynden , hvorledes knnde han da borttage andre Folks Synder . Sandelig ! Jomfru Maria holdes i storre Agt og Mrc hos Tyrkerne og Hedningerne, end hos disse uchristelige Ugudelige ! " „ Det have , " tog den nnge Mand Ordet , „ maastc nogle munzerffe Sværmcrc og ugudelige GjeNdsbcre gjort , mc » aldrig nogen tro evangelisk Christen . " „ Gjor Eder ingen Umage ! " sagde Smeden rolig : „ Med saadanne Kjcrmpcrc kan ingen crrlig Kamp sinde Sted . Jeg har bedre Vaaben mod saadanne Folk og venter blot paa.den rette Tid dertil . Jeg onstede blot , at den maa komme snart . " „ Og vil I ncrgte , " vedblev Staldmesteren ivrigt og ligesom i en Sciercns Lob : „ Vil I ncrgte det , da Alt ligger klart for Dagen ? Hvad er der fremgaaet af eders Luthers og hans Fcrllers ugudelige Lcrre , Prædiken , Disputation og Skrift ? Ligesom der af en mudret Dynge eller Msdding blot fremkryber

<h4>962</h4>ja med Skræk og Forfcerdelse , da var Døden rigtignok en Ulykke . Mcn den Christnes Die ser ganste anderledes paa denne sidste Fiende , der tilintetgjøres i Døden . Bliktet paa Graven og Forraadnelsen er . for det christelige Sind et Blik i Guds vidunderlige <b>Visdom</b> og Kjærlighed , der endnu i Doven velsigner og gjor vel mod Begge , baade mod dem , som do , og dem , som blive tilbage , Christus har taget Magten fra Døden ! " „ O , dog er Døden ogsaa en Herrens Dom ! " svarede Leonardo med dyb Alvor . — « Jeg ved ikke , hvad det er for en Skræk , som netop gaar tgjenncm min Sjæl- Margrethe, ak , jeg er bleven saa bange , saa saare bange , for mig selv , for Dig , for min Fader ! Margrethe , sig mig , om Herrens Dom i min Faders Død var til Livet eller til den evige Død ! Forstoder ikke Gud ogsaa ved Døden ? Er det trostcligt , naar vi maa sige os selv , at med Legemets Død ogsaa Guds sidste Naades.Wieblik er forbi ? Og vi maa sige os det , fordi Gud ei kan drive noget Spil med sin Retfærdighed- Den hele Verdensstyrelse , al evig Sandhed , som gaar opvcektende , helligende igjenn ' em det hele Mennestefjsn , oplostr sig i et toMt Taagedillede , hvis Guds Retfærdighed ei er Sandhed. Gud holder , Gud inaa holde sin sidste Dom ved Døden . Ak , hvem siger mig , at min kjære , dyrebare Fader er domt til Livet og ikke til Døden ! " „ Din Tro paa Herren og paa hans Forjettelse ! " svarede hans <b>Kone</b> . „ Har Du ikke sagt mig . at din kjære Fader af ganste Hjerte har hengivet sig til Christus ? Hvilken Trost havde jeg med Hensyn til min Faders Død , hvis jeg ei vilde tro Christo ? Ja , min eneste Ven , lader os ikke klage og ssrge , som de , der ei have noget oaab ; men glæd Dig med mig over de Forlostes Salighed ! Hvor vederkvægende , hvor uudsigelig herlig er ikke Tanken paa den forventende Salighed , paa en Forløsning fra alle Onder , en Nydelse af den fuldkomneste Lykke ! Bliver itke ved et saadant Haab endog den haar « deste og ssrgeligste Tilvcerelse bersvet oens bittre Braao ? Maa ikke den sidste Smertenstaare borttsrres , naar Hjertet er saligt i Herren ? " „ Skjcend kun paa mig , min dyrebare <b>Kone</b> ! " svarede Leonardo . „ Skjcend paa min Vantro , min svaghed ! Ved dm Faders Dødsseng og Ligkiste var jeg en god Trøster , og havde fuldtop deraf ; men nu er jeg paa engang fattig og tom , og Vantroen vil gjøre mit Haab tilstamme . Du vil , at jeg stal være salig med Dig i Haabet ; men vt haabe kun , hvad vi ikke se , og hvorom Ingen fra sin Grav kan give den Levende Kund , stab . Hvis det nu ikke var sandt , hvad vi haabede ! Hvis den forventede Salighed vel var en stjon , men dog bedragerisk , forfængelig Drsm ? Hvis nu den Ene blot var den Andens Eftersnakker i denne Henseende , fordi Enhver har sit Velbehag deri ? " „ Stakkels Mand , hvorledes Du plager Dig ! " raabte Margrethe . „ Det forundrer mig , at Du ikke moisommeligt samler Alt , forat bevise for Dig selv . at vi ikke evigt do , men engang virkelig ville blive salige ! " „ Hvad vil Du sige dermed , Margrethe ! " spurgte Leonardo , „ Nu , " vedblev hun smilende . „ Vkal jeg bevise det for Dig , at vt engang ville leve evigt og i Sandhed og Virkelighed blive salige ? " „ Bevise ? " gjentog Leonardo forundret . „ Hvorledes vil Du gjøre det ? " „ Et . " svarede hun . næsten i en glad Tone , „ Du er bleven en Vantroende , og gjør , som Thomas , der blot vilde tro hvad han kunde se . Nu lceg Marte til , hvad jeg siger , Leonhard ! Jeg siger Dig : Sjælen vil og maa leve evigt ; thi den k ' an ikke do , fordi den ri har nonet Leycme , men er en <b>Aand</b> . Men Aanden dsr ikke . Nu , Leonhard . hvad siger Du til denne fortræffelige Trsst ? Er Du nu tilfreds ? Har Du nu Tro igjen ? Er ikke det en ganste scerdeles dyb <b>Visdom</b> ? " „ Du spotter mig ! " svarede Leonardo . „ Det kan være en meget lærd Tale ; men for den ssrgende og tvivlende Sjæl er den et Luftkastel . Bevis mig først , at jeg er en <b>Aand</b> . " „ Nu , " vedblev Margrethe venligt smilende : „ Jeg kan endnu paa en anden Maade bevise Dig , at vt ikke evigt ds. Ser Du , min Leonhard , Alt maa blive fuldkomment , ellers har det ikte noget <b>Maal</b> eller nogen Ende . Naar Du planter et Trce , saa er det ikke nok , at det frembringer Grene og Blade , men det maa vore og styrke sig saalænge . indtil det fremdriver Blomster og værer Frugter og er blevet et fuldkomment Trce .

<h4>970</h4>er netop opfundet af saadanne taabelige Mennesker , som jeg selv er ? Hvorledes vilde Du da helbrede min Vantro og bekjæmpe mine Tvivl ? " „ Nu du blinde Mand ! " raabte den Blinde med glad Rorelse og strakte sine Hænder ud for at finde og gribe sin Mands Haand . „ Foler Du da ikke , at jeg selv spotter over al min <b>Visdom</b> , med hvilten jeg har villet fore Dig til Troen ? Foler Du ikke tgjennem din hele Sjæl , hvorledes Gud gjor denne Verdens <b>Visdom</b> til Daarstab , og tlltntetgjør de Vises <b>Visdom</b> , naar de ville udgrunde og bevise det Himmelske og Evige ? Siger ikke dit hele Hjerte Dig , at Du er forladt , udkastet i den ode Drken , saasnart Du slipper Troen . Hvorledes vil Du med din egen <b>Visdom</b> igjenfinde det , som er gaaet tabt for Dig , naar Du ikke lamgere tror Guds Ord ? Intet og Ingen paa Jorden kan lære Dig at tro , uden Guds hellige <b>Aand</b> alene , der oplyser dit Hjerte , foreholder Dig din Livs . og Hjertensd , lader Dig stue ind i din Elendigheds hele Dybde og driver Dig op til Herren og Hjælperen ud af al Legems « og Sjælensd . Min dyrebare, kjære Ven , " vedblev Margarethe bevæget : „ Jeg ved , at Du ikke alvorligt tvivler , at kun dit barnlige Hjertes Smerte over din dyrebare Fader , som dit legemlige Die ikke mere skulde se , piner Dig med saadanne Tanker ; jeg ved sikkert , at Du meget snart , førsonet med din Gud og Herre , igjen vil finde Dig tilrette i den evige Kjærltghed og Naade , — men erkjend det med mig paany , — mangle vi Troen , da ere vi fortabte , og al Verdens <b>Visdom</b> kan ikke lægge et Korn Trost og Fred i vor arme Sjæl . , Thi Ordet og Korset er med engang en Daarlighed for dem , som skulle blive fortabte ; men for os , som tro , er det en Guds Kraft til Salighed . Vel os , min sode , kjære Ven , at vi have en anden Borgen for vor Salighed , end denne Verdens <b>Visdom</b> , nemlig vor Herre Jesus Christus , om hvilken vi vide , at han er kommen til Verden , for at gjore Synderne salige . Naar denne min og din Herre for mig og Dig beder : Fader , jeg vil , at hvor jeg er , stulle ogsaa de være , som Du har Hivet mig , paa det at de kunne se min Herlighed , som Du har givet migi saa staar jeg med de to Ord : jeg vil ! Alt ned , hvad der kunde reise sig derimod . Naar min Gud , min Herre og min Forlsser siger : jeg vil , saa er ingen Tvivl mere mulig ; thi hvad min Herre siger , det er sandt , og hvad min Herre vil , det ster , og mig tilkommer det , at jeg tror derpaa . Og medens de Andre pine og plage sig , og komme dog ikke ud af sin egen <b>Visdom</b> , sidder jeg rolig og trostig under den Hoiestes ' Beskyttelse . Og naar jeg tror min Herre , naar han siger : Gud rc- Aja > rligl ) eden , saa tror jeg ham ogsaa , naar han siger : Jeg vil at Du skal va > re hos mig ! " Da Margaretbe her taug , sank Leonhard med bævende Taarer i Diet til hendes Bryst og sagde . „ Jeg arme , arme Menneske , der vilde undlobe min Guds baand ! Ak Margarethe , jeg har bestandig troet , at Gud har givet mig Dig som en god Engel til at lede og fore mig paa min Livsbane ; men fra idag ved jeg , at Du for mig er en Guds Gesandt ! " „ Synd ikke med din Kjærlighed ! " bad hans <b>Kone</b> og græd af Glæde ved hans Hjerte . Og den gamle Mester Urban traadte ind og sagde : „ Born , jeg har hort Alt / hvad I have sagt . Og nu , da dit Hjerte igjen er trostet og glad , min Son , saa erfar da og lces det her i din salig Faders sidste Skrivelse , som han har ladet afgaa til mig tre Dage for sin Død : Du stal blive hos os , opsiaa dit Hjem i vort kjære , tydsse Fcedreland; mit Hus stal være dit Hus ! Forat Du , min Son , skulde glad indvilge i dette , er din kjære Fader dod glad og salig ! Hvad vil Du nu gjore ? " „ Guds Villie sie ! " svarede Leonhard og bad stille og med saligt smilende Ansigt .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013042608043'target=_>, 1876, Aktstykker til Belysning af Forholdet mellem Biskop Schreuder og det Norske Missionsselskab, fra Biskopens Ordination indtil Generalforsamlingen 1873</a><h4>540</h4>Tilfcelde er omtrent umuligt at gjennemfore en for faa lang Tid , som nodvendigt , uafbrudt og sammenhængende Underviisning , thi snart f . Ex . hore de , at en Gjed eller et Faar eller vel endog et Stykke Kvceg er at faae tilkjobs et eller andet Sted omkring i Ncerheden , og snart er et eller andet Distraherende paafcerde . og saa er det : at beherske sig selv ved en saadan given gunstig Anledning ganske unationalt , saa kommer da en Uroligheds <b>Aand</b> paa dem , og det er oftest frugtestlost at gjore Indvendinger og soge at tale dem tilrette , naar forst noget er kommet dem i Hovedet . Skjondt nu Lukas har nogen <b>Begavelse</b> og iscrr stor Frimodighed og Uforfcerdethed, saa ere hans Evner ikke saadanne , at han overhovedet lunde modtage nogen videregaaendc <b>Uddannelse</b> ; han har allerede udtomt sig ligeoverfor sit forste Kuld af Skoleborn , der nu i boglig Kunst er langt lamgere komne end han ; og ved Forsog vaa at bringe ham lcrngcre frem i visse <b>Kundskaber</b> , synes hans Evner at stoppe op pludseligeu ved en bestemt Gramdse . Thomas fra Umpumulo derimod har uncegtelig gode ja sjeldne Evner og synes bekvem for en langt videregaaende <b>Uddannelse</b> , men heller ikke hos ham spores synderligt af Selvdrivt og af given stadig Flid , der er saa uomgjengelig nodvendig , for at Undervisningen kan gaae noget ftodt og stadigt fremad ; derhos cr ogfaa han inde i Familieforholde og andet faadant, der distraherer og standser Fremskridtene . Nu har han forladt sin gode Situation her og er vendt tilbage til Umpumulo , fordi hans <b>Kone</b> det Fjog havde jeg ncer sagt lcengtes saa efter sine Slcegtninge over vaa Umpumulo ( blandt dem en Hendrik , der formedelst Horeri med Unokutemba forlod os og gik til Amerikanernes indfodte Prcedikant Umbijane , gift med vor forste Dobte nu ogsaa amerikansk Christen ) . Uagtet Thomas staaer over Lukas med Hensyn til Evner , saa besidder han dog ikke denne sidstes Uforfcerdethed i at trcede Farer , forbundne med Sandhedens Bekjendelse , uuder Vine . Vi have ogsaa havt et andet og yngre Emne i Vie , nemlig Samson fra Umpumulo , nu lamgere Tid i Tjeneste hos Kyllingstad . Paa et Mode af Indfodte ifjor vaa Umpumulo optraadte ogfaa han med sit Pund , som bestod i et bekjendende og formanende hjerteligt Ord , der hovedfageligen gik ud paa , at de christne Unggutter syntes at ballancere saa taaleligen godt i Sandheden og som Troende , indtil det kom til Frierhistoner , men-der tabte de som oftest Ballancen og forholdt sig i Skind og Sind ikke bedre end hedenske Unggutter .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009061903026'target=_>Allen, C.F., 1864, De tre nordiske Rigers Historie under Hans, Christiern den Anden, Frederik den Første, Gustav Vasa, Grevefeiden : 1497-1536</a><h4>145</h4>ved sine rige Indtægter og sine Besiddelsers Storhed , langt mere end den , som bar Kongenavnet . Da Albrecht omsider prøvede paa at bryde Aaget , gik det ham som hans Forgjængere : de Store gjorde Opstand og overdroge Thronen til en Anden . De gave den til Margrete , uden Tvivl i den Mening , at de ikke vilde have mindre let Spil under en <b>Qvinde</b> end under en Mand . Disse Forhold maa man vel holde for Øie , naar man vil forstaae Unionstiden og bedømme de Styrende retfærdigt . Naar Alt var opløst , naar Magt var bleven til Ret og Lovene ustraffet traadtes under Fødder , naar Kronen var nedværdiget og Stormændene uenige om alt Andet end det , ikke at ville lyde Nogen over sig : saa var det en Daarlighed at vente , at med Unionen skulde alle onde Lidenskaber dysses , Fred og Samdrægtighed herske , borgerlig Orden trives under Lydighed under Loven ; og en Uretfærdighed at paastaae , at det var Unionskongerne , som ved deres Uduelighed fordærvede Alt og forspildte Unionens Fremtid . Kun een af dem , Erik af Pommern , kan med Rette kaldes uduelig ; de øvrige vare , om end ikke udmærkede Personligheder , saa dog heller ikke saa slet udrustede til deres Kald , at de jo uden Vanhæder kunde have baaret deres Krone , hvis ikke Forholdene i Sverrig havde været saa vanskelige , at der hørte en Viisdom og Kraft til for at magte dem , som , naar den findes , maa regnes blandt de sjældne Undtagelser , og som man ikke kunde vente skulde gjentage sig i en heel Række af Fyrster , der afløste hinanden paa * Nordens Throne . Det var som et Vidunder , og et stort Vidnesbyrd om Margretes overlegne <b>Aand</b> , at hun for en Tid formaaede at kue de onde Aander og med sin Qvindehaand føre den ustyrlige og i meer end hundrede Aar til enhver Ave uvante svenske Adel til det <b>Maal</b> , hun vilde . Hun havde imidlertid en Medhjælper

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014052208029'target=_>Daniels, W.H., 1877, D.L. Moodys Liv og første Virksomhed i New York og Chikago</a><h4>53</h4>men hun var den eneste Person i Verden , som nogenfinde kunde såa Bugt med ham . Han var stolt og egensindig i hoieste Grad , men fuld af ædelmodige Planer . Han dar uregjerlig , tildels sordi han af Naturen var anlagt til selv at vcere Leder . Hans Karakter var dog ikke ondskabsfuld . Dersom man kuude bringe ham til at indse , at han havde gjort Nogen Uret , var han villig til at bede om Forladelse og opfore sig bedre for Fremtiden . Den sidste Vinter , han gik paa Skole , var han i sit syttende Aar . Han var den ledende <b>Aand</b> blandt Drengene , og saa mange Sksierstreger oplcerte han dem til , at Lcereren tilslnt blev rent fortvivlet og trnede med at <b>vise</b> ham ud . Dette var eu stor Sorg for hans gode <b>Moder</b> . Hun sagde ham , hvilken Skam det vilde vcere for hende , om en af hendes Ssnner blev udvist af Skolen , og randede ham til at gaa til Lcereren , bede om Tilgivelse for fin slette Opfsrfel og soge at gjore sin <b>Moder</b> Wre istedetfor Skam . Dette gjorde han as et oprigtigt Hjerte , og lceste for fsrfte Gang i sit Liv med virkelig Flid Resten af fin Skoletid . Men det var for sent for ham at drive det til Noget i den Retning . Den Tid var kommen , da han solte sig kaldet til Livets strenge Arbeide , og med den Smule <b>Kundskab</b> , han havde samlet sig han vidste neppe selv hvorledes maatte han gaa nd og dristigt begynde sin Kamp-med Verden . Hvad han i Barndommen havde erfaret af rcligisse Indtryk , syntes nu at vcere forsvuudet , og han var voxet op til en ung Mand eller rettere til en stor Dreng , uden anden Gudsfrygt end Kjcerligheden til sin <b>Moder</b> og en kraftig Beslutning om at blwc en retskaffen , virksom og forfremmet Mand . Han havde

<h4>71</h4>Med den fllmme Begeistring i det Aandelige som i alt andet begyndte han snart at tale i Bonnemsder i Kirken om hvad Gud havde gjort for hans <b>Sjel</b> , og stundom tilfsiede han nogle Formaninger , fom slet ikke var fmigrende for de fornemme Menighedslemmer , der fad rundt omkring ham ; de tog det altid meget ilde op. En venlig <b>Dame</b> gik til hans Onkel Samuel ug bad ham raade den unge Nyomveudte til at tie stille indtil han var kommen faavidt , at han kunde tale til Forsamlingens Opbyggelse . Men Onkelen svarede , at det glcedede ham , at Dwight havde Mod til at bekjende sin Tro paa Frelseren i en saadan Forsamling; han vilde ikke lcegge ham et Halmstråa i Veien . Snart efter begjcerede han at blive Medlem af Mount Veruonkirken og gik til Forstanderne for at lade sig prsve i Tro og Kristendomskundskab . Men han havde nok ikke lcert faameget , at han kunde staa sig ved en faa streng Overhoring . Han havde nok Tro , men med hans <b>Kundskab</b> stod det daarligt til . Den Helligllllnd havde begyndt sit Arbeide : ham til Helliggjsrelse , men den orthodoxe Lcrre havde ikke gjort synderlig Indtryk paa ham . Han kunde ikke besvare de gode Forstanderes Spsrgsmaal ; men saavidt han kjendte sine Pligter mod Kristus , Kirken og Verden , var det hans alvorlige Vilje og brcendende Onste at opfylde dem .

<h4>142</h4>troede han , de vilde gjore sig selv til dannede og dygtige Mamd og <b>Kvinder</b> . Med denne Tanke for Oie blev der anvendt liden Tid paa Vibelens Geografi eller Oldtidsminder , men Evangeliet om Gnds Son , rent og simpelt , blev paa enhver mulig Maade lagt paa Bornenes Hjerter . Medens saaledes andre beslægtede Anstalter passende kunde kaldes Bibelskoler , var Missionen paa Nordtorvet fremfor alt en Evangelieskole. Den tog naturligvis denne Retning paa Grnnd af lederens <b>Aand</b> og Erfaring , der var besluttet paa Intet at vide blandt dem ' uden „ Kristus og Ham torvfcestet . " Denne Veslntning var for ham let at ndfore , thi han var grundigen omvendt til og ganske godt bekjendt med Kristus , medens hans andre <b>Kundskaber</b> i Sammenligning hermed vare faa oa übetydelige. Hans Ven Stillson erklærer , at i disse Aar ved han ikke om , at Moody cicde nogen anden Bog end et Exemplar af det nye Testament . Ingen Mand var mere hungrig efter at loere end han , men hans Smag gik helt og holdent i Retningen af , hvorledes hau skulde arbeide for Kristus . En Fortælling havde Interesse for ham hovedsagelig som en Kilde / hvoraf han knnde hente Oplysninger om Evangeliets Sandhed, et Digt havde liden Betydning for ham , medmindre det kunde blive smiget i hans Skole- i Videnskab og skjonne dunster havde han ikke nogensomhelst <b>Kundskab</b> . Meu i Maadeu at arbeide og virke var han i hm Grad lcerd- , dette blev saa fnldstamdig erkendt, at mange dybsindige Lcerde og Doktorer i Theologien stjsut de stodtes over haus daarlige Rhetorit og endnu slettere Grammatik henvendte sig til ham for at modtage Undervisning uni de Mander man maatte gaa frem paa for at naa og frelse den

<h4>144</h4>store forsomte Masse af Syndere , der stedse svcermcde om dem . „ Dm , sum vinder Sjcele , er <b>vis</b> " , siger den hellige Skrift , og Mlllllt med dette <b>Maal</b> kunde nmn ikke blandt det lcrrde Loegfolk eller Geistligheden i alle de Forenede Stater finde en visere Mand end den raa , noget fremfufende og vel djerve Leder af Missionsskolen paa Nordtorvet . I den Hensigt at lsfte Gjerningen og samle Kullene til en stcerkere Flamme , foranstaltede han ugentlige Aften-Bonnemoder i den gamle Salon . I en Henseende var disse Nsnnemoder eiendommelige ; de vare ueinlig intet andet end en Forsamling af Mennesker , der snstede noget af Herren , og som kom sammen i den Forventning at erholde det ved at bede . Disse Mennesker en Samling af bodfærdige og angerfulde Syudere kom sammen , ikke fordi de maatte , men fordi de snstede at komme . Til dette mindre pyntelige Sted , hvor Scederne bestod af uhovlede Planker, lagte ovenpllll tomme Spigerdunke , som blev oplyst af nogle faa Lys og beskyttet mod Voldsomhed af Politiet , kom disse Born og celdre Mennesker , hvis Hjerter Evangeliet havde naaet , for af Moody og hans nye Testament at lcere , hvad de havde at gMe for at blive frelste . . Der var en hoist tiltalende Livlighed i Bsnnen oq Talen ved disse <b>Moder</b> Noget , man kunde vente af Skolens hele Methode . Disse Bedende og Ssgende troede paa Gud , ikke som en „ ukjendt og uhendeliq Magt " , men som Himmelens og Jordens Skaber og Styrer . De troede paa lesns som en „ reformerende Iode " , men som Guds der kom til Verden for at frelse Synder « ' : " g , siMt de ikke havde noget klart Begreb om den He lkgaand , vare

<h4>549</h4>. . Mennesket lever ikke af Brsd alene , men af hvert Ord , som udgaar af Guds Mund " , sagde Moses i sin Afskedstale til Israel . Gud var i Begreb med at aabenbare sin Tjener sin skjulte <b>Visdom</b> paa en saavel kjcerlig som eftertrykkelig <b>Vis</b> , og det med den Fslge , at han blev en af fin Tids heldigste Bibelfortolkere . En Lsrdag , da han stod fcerdig til at forlade sit Hjem for at bivaane et Mode langt bvrte den fslgende Dag , fagde han til fin <b>Hustru</b> : „ Jeg har modtaget ' > > ' fra Harry Moorhouse , en Englænder, som lalder sig . . den unge Prcrdikant " , og som underretter mig om . at han kommer til Chikago idag og vil prcedike for mig , derfom jeg onster det . Det er forfent at faa nogen Anden , faa jeg tcenker vi maa lade ham forfoge om Formiddagen , men dersom det gaar uheldigt , maa Du bede Medhjælperne at faa fat i nogen Anden til Aftengudstjenesten , eller lade dem holde et Bsnnemsde " . Da han kom tilbage ncefte Uge fpurgte Moody ivrigt , hvorledes det var gaaet Harry Moorhouse som Prcedikant : „ Han skilte sig mcerkvcrrdig godt fra det " , var Svaret . „ Soudag Formiddag talte han over den Text : „ Saa

<h4>575</h4>Henseende bliver mere og mere aabenbar , og Kirken bescetter undertiden sine hsieste Embeder med Mcend fra andre Lande , ikke fordi Amerikas Præstestand mangler Gudsfrygt , <b>Dannelse</b> eller Veltalenhed , men fordi den mangler en grundig Indsigt i den hellige Skrift , og iscer ikke forstaar den sceregne Anvendelse deraf , fom kunde kaldes „ Udlceggelses-Prcediken " . Andre Evangelister ere ogfaa i den senere Tid blevne Mcend af „ den ene Bog " , og den Kjendsgjerning , at disfe Mennesker , der er opstaaede af Folkets Midte og ganske manglede lcerd <b>Uddannelse</b> , men ere mcegtige i Skriften , har vceret hcedrede af Gud til at fsre flere Sjcele til Ham end nogen anden nulevende Klasse Mennesker , er et nyt Bevis vaa Sandheden af Frelserens Udsagn : „ De Ord , som jeg taler til Eder , er <b>Aand</b> og Liv . " Dette havde Moody tildels lcert af sin Ven Moorhonse, men det skulde brceudes ind i hans Hjerte med Bogstaver af Ild . En stor Del af Aaret 1871 led han under store Sjcrlskampe med Hensyn til sig selv og sin Gjerning . Han Pleiede at grcede og bede med stor Angst i sit Lsnkammer , og derfra gaa ud til fin Dagsgjerning med nedslaaet Ansigt . Han opfordrede stadigt sine Venner til at bede for ham . Efterat have gjort Bekendtskab med to meget gamle <b>Kvinder</b> , som vare bekjendte sor sit stcerke Troesliv , tiltrods for store Prsvelser, gik han ofte til dem som et forgrcedt Barn og bad dem lcere ham at stole fuldkommen vaa Gud . Hvad der iscer plagede hans Sjcel var Tanken vaa , at han var en uvidende Mand , medens han dog blev seet op til som en acmdelig Lcerer . Han begyndte at uudres vaa , om han skulde vcere en af hine blinde Veiledere , fom Herren omtaler , og om han ikke en

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111400046'target=_>Funcke, Otto, 1878, Reisebilleder</a><h4>126</h4>Men vi ere i Salzbrunn og ikke paa Overtroens Vloksbjergl Det er jo til en <b>vis</b> Grad ganske i sin Orden , at de fleste Badegjester saa blindt folge Legens Forflrifter , men denne blinde Lyoighed er et Bevis for , hvad et Menneflebarn kan finde sig i , naar man mener at kunne flasse en svoekket Konstitution ny Livskraft . Livet ved et Badested viser bedst , hvilke Opofrelser man kan paalcegge sig for at naa dette <b>Maal</b> . Mcend , som helt og holdent leve for sine Forretninger , som hverken Sendaa , eller Hverdag have Tid at ofre Guds Ord en Time , hvem <b>Hustru</b> og Bern aldrig kunne faa itale uden ved Maaltiderne , de rive sig les for Kurens Skyld baade i Uger og Maaneder . Forsagte <b>Kvinder</b> , emme Medre lade LEgtefelle og Born skjette sig selv og gjere lange , lange Badereiser . Gjerrige , der ikke ere istnnd til at ofre en Hvid for Uddredelsen af Guds Rige eller for Menneflehedens Interesser , de udgive her uden Knurren den ene Sum efter den anden . Fattige Familier flrabe sammen alt , hvad de eie , eller hvad de kunne laane , for at kunne betale en Badereise , og fele Trykket i mange Maaneder , efter at den er endt . Fraadsere afholde sig her fra sine Nydelser ; den forvente Gane kan gjere saa mange Indvendiuger , den vil , den maa underkaste sig Kurdicetens drakonifle Love . Syvsoverne udfere utrolige Ting : de spcege sit dovne Kjed , forlade Kl . 6 om Morgenen sin blede , varme Seng for at vcere ved Sundhedskilden paa Klokkeslaget . Dedsmatte , elendige Staller , der neppe formåa at flebe sig frem , gaa med afmaalte Skridt den bestemte Tid — ikke et Minut for meget eller for lidet . Forfengelige <b>Damer</b> gjere Afkald paa sine lette , smagfulde

<h4>158</h4>Man skulde vel med Nette vente , at der var Trcengsel om lefusbrsnden , men med Undren og Skrcek se vi , at der er ncesten ganske tomt . De samme Mennesker , som have kunnet taale store Savn for at opnaa en kortvarig , tvivlsom , legemlig Bedring , som villigt have korsfestet Kjedet og afholdt sig fra syndige Vaner , disse have ligesaavel som Andre de mest intetsigende Undskyldninger paa rede Haand , naar det gjcelder at modtage Indbydelsen til den Sundhedskilde , der alene og i Sandhed fortjener dette Navn . Man udscetter , man vil vente , idag har man ikke Tid , imorgen ikke Lyst , snart er man altfor rig eller altfor fattig , for frist eller for syg , for ung eller for gammel , faa skylder man paa Fornyelser , paa sit Kjebmandstab, eller man har taget en <b>Hustru</b> tilcegte og kan derfor ikke komme . Men Sjcelen fortsrreS , det indre Menneske skrumper md som en « egyptisk Mumie , ingen Kunstens Midler , Sminke , Vellugt , Perler eller Guld kunne skjule Dedens Tegn , al hyklet Glcede kan end ikke for Ens eget Bre overdeve <b>Sjelens</b> Angststrig , vor Tids fine <b>Dannelse</b> og stolte Oplysning kan ikke forjage Dedsfrygten . Den indre Armod stikker frem gjennem dm synlige Rigdom ; man taler om « den skjenne Verden hinsides Graven " , om den « kjcere , gode Gud i Himmerige", om et « edelt , godt Hjerte , om en retstaffen Vandel ; men lytter man til Sandhedms Rest i Sjcelen , stal man erfare , at derinde blede dedelige Såar , for hvilke Verden ingen Lcegedom har , og at vi trcenge en ny Aano og et nyt Hjerte — just hvad Jesus byder .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012041608169'target=_>Krummacher, F.W., 1856, Thisbiteren Elias</a><h4>391</h4>vil omvende sig til Herren , og reise sig op , hvcrmcget man cnd prcrdiker for det . Vrodct olivcr tagct sta det , og givct Hundene , som smcrgtc ndcnfor . Hcrom advarer cgsaa den Herre Jesus , idet han siger : Men i Tandhcd siger jeg Eder : ~ Dcr var mange Enker i Israel i Elia : Dage , der Himmelen var lnkkct i tre Aar og ser Maancder , men til Ingen as dem blcv Elias sendt , ndcn til Zidons Zarcpta , til cn Enkeqvinde " ' ) . — Der siddcr han nn sornoiet med det hele Huns . 31 l Nod er sorsvnnden . Melct i Krukken blivcr ikkc fortcrrct , og Olickrnsct fyldes altid paany af sig seiv isolgc Herrens Forjcrttelse . Det aandcligc Taffel bliver ogsaa altid igjen dcrkket paany . O , hvcr den arme <b>Qvinde</b> glcrdcr sig over , at hun saaledcs hvcr Dag kan siddc ved denne <b>vise</b> , ftommc Mands Foddcr , scr as ham at modtage Undcrviisning cin de himmelske Ting , og bvor gjerne oplader Elias sin Mund til denne clskclige , hjertcligc og cnsoldigc Tostersjcrl. De bede sammen , sammen i Moses og Prophctcrne , nnderboldc sig med hinanden om den sorjcrttede Messias og hans Komme , istcmmc nu og da en aaudelig Tang til Herrens 3 ( sre , og Tiden blivcr dem aldrig lang , og Guds Engle knndc glcrdc sig over en saadan lillc Kirke i ørkenen , og med Lvst stue ned til den . Tcer , seer , Vrodre , det er Udgangcn og Enden paa en Vei , so-n begvndtc i saa dvbt Morke . Taaledes sorer Gud sine Born , i det Legcmlige som i det <b>Aandcligc</b> altid til det hcrligste <b>Maal</b> .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010070606083'target=_>Schiern, Frederik, 1875, Nyere historiske Studier</a><h4>1475</h4>disse Ord ere tagne , sendte hun det til sin Mand , som paa denne Tid efter Murrays Død var bleven Regent i Skotland , og i det fortrolige Svar , som Jarlen af Lennox i denne Aiiledning tilskrev sin <b>Hustru</b> , yttrede han , at han var overbeviist om Marias Skyld ikke blot ved sin « egen <b>Kundskab</b> , men ved hendes Haandskrift , Bekjendelsen af Folk , der vare gaaede i Døden , og anden ufeilbar Erfaringa ! . Jarlen , der vistnok kjendte Marias Haandskrift , sigter ogsaa her til Sølvæskens omstridte Aktstykker . Han vedbliver saaledes : » Det vil være en lang Tid , som vil være nødvendig til at bringe saa notorisk en Sag i Forglemmelse , til at gjøre Sort til Hvidt eller <b>vise</b> Uskyldighed der , hvor det Modsatte er saa vel bekjendt . De meest Upartiske , derpaa forlader jeg mig , tvivle ikke om Retten af Eders og af min Sag og om den retfærdige Anledning til vor Ugunst . Hendes riglige Pligt mod Eder og mig , som de deri interesserede Parter , vilde være en sand Bekjendelse og hendes uskrømtede Anger over hiin sørgelige Gjerning , lige saa slyg for hende at tale om , som sørgelig for os at tænke paa . Gud er retfærdig og vil ikke i Længden lade sig bedrage ; men som han har aabenbaret Sandheden , saaledes vil han straffe Uretten « . Efter at Jarlen af Lennox imidlertid under den fortsatte Partikamp var bleven dræbt ligesom Murray , og havde faaet Jarlen af Morton som Efterfølger til Regent i

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008090303002'target=_>Becker, Karl Friedrich, 1842, Karl Friedrich Becker's Verdenshistorie</a><h4>606</h4>man kncrlede for Billcdeme , man drev ogsaa overtroisk Trolddom med dem : man kradsede noget af Farven af dem , og kom det i Vinen ved Nadveren ; <b>Modrcne</b> lagde deres nyfodte Born i Armene paa mdviede Statuer , for at gwre dem deelagtige i Helgenes Velsignelse ; Syge gnede deres Bind og Tepper ftaa dem , for at blive friske . En saadan Tjeneste maatte rigtignok forekomme som « overensstemmende med Chriftendommens <b>Aand</b> og som et Tilbagefald til et lavere Trin af religws Sevidsthed . Keiser Leo vilde standse dette Uvcesen ; han havde ofte maattet hore , hvorledes loder og Muselmcend bebreidede Christendommcn denne Billeddyrkelse , hvorledes denne endog stod i Veien for deres Omvendelse, og han havde foresat sig , med al Kraft at rydde denne Forargelse af Veien . I Førstningen lod han blot Billederne stille hoiere , for at unddrage dem den umiddelbare Berorelse , og forbyde deres Dyrkelse ; men da Indbyggerne paa de cykladiste Ver gjorde Ovror derover og med Vaabenmagt maatte bringes til Lydighed , udkom der et almindeligt Forbud imod alle Billeder af Christus , Englene og Helgene , og der blev truet med svcer Straf for Overtra ' derne . Germ anus , Patriarchen i Konstantinopel vcegrede sig ved at underskrive dette Edict . Han nedlagde offentligt sin Vcerdighed , og der opstod nu i Riget to Partier , Billeddyrkerne ( kixo ^ oso ^ vl ) og de talrigere Billcdstormere (k/xo^ox).»---<77«1), hvilke bekcempede hinanden ofte med blodig Forbittrelse . Da det ene Parti med halsstarrig Heftighed holdt fast ved det Gamle , forglemte det andet ogsaa <b>Maal</b> og Oiemed , og i den heftige Kamp imellem to Partier , der hvert paa sin Side vilde drive det Uderste igicnnem , gik , som det saa ofte er Tilfcrldet i menneskelige Anliggender , det rette Synspunkt meer og meer tabt . Thi ogsaa Billedstormerne kunde man med Rette bebreide , at de i Kampen imod Misbrugen , angrcb Sagen selv i en kold forstandig , til jodisie og islamitiste Abstractioner tilboiclig <b>Aand</b> , og vilde skille Menneskeheden ved en af dens cedleste Goder , ved Gemyttets Oploftelse igiennem den guddommelige Kunst . Imidlertid gik Pave Gress or 11. vel neppe ud fra den Anskuelse , da han fordomte hine Beslutninger imod Villederne . Folket i Italien var paa hans Side ; Romerne gjorde Opstand , i Ravenna seirede ogsaa Venncrne af Billederne , Erarchen blev drcebt i en Tumult , og Staden kom derved i den longobardisie Kong Luitprands Hcender . — Han regierede fra 712 — 743. Leo efterlod sin Son Constantin V . Kopronymus * ) Niget vaklende saavel indvortes som udvortes .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2007091102001'target=_>Lassen, Albert, 1831, Lærebog i den almindelige Historie</a><h4>968</h4>og Hildebrand , som i det Virkelige allerede saa lcrnge havde regjeret Kirken , blev nu Pave , oh antog som saadau Navnet Gregor den syvende . Fra nu af ar , beidede han med al den Kraft og urokkelige Stand « hastighed , som hans faste Villie og Overbeviisning om redelig Hensigt gav ham , paa at fuldfore sit Vcerk , som var intet Mindre end at skaffe Kirken fuldkommen Uafhængighed og ophsie den over al Indflydelse af hvilkensomhelst verdslig Magt . Han gik ud fra- den Idee , at stulde Paverne vcrre Chrisii Efterfslgere , saa maatte de vcere Forstandere for hans Rige paa Jorden , det er den hele Christenhed , og deres hsieste <b>Formaal</b> maatte vcrre at hceve Menneskeheden til Sædelighed og « andelig Reenhed, og dette hsie <b>Formaal</b> maatte vcere overord : net alle andre jordiske og politiske Hensyn . Kir , ken , saa yttrede han , maatte vcere fri , Alterens Tjenere lssrives fra alle verdslige Baand . Men ligesom <b>Sjelen</b> ei kan vcere virksom uden Legemet , saaledes ei heller Kirken uden verdslig Gods og Eiendom ; at bevare den dette er dens Pligt , som fsrer det verdslige Svcerd , Keiseren . Derfor ere Keiseren og Fyrster nsdvendige for Kirken . Ver ? den ledes af tvende Lys , Solen og Maanen . Den apostoliske Magt er at ansee som Solen , den keij serlige , kongelige og fyrstelige som Maanen . Ligesom Maanen laaner sin Glands af Solen , saaledes Fyrsierne af Paven . De have deres Magt kun af ham , thi han har sin af Gud , og ssylde ham Ly » dighed . Alle Kirker ere den romerske underdanige , som Dettre deres <b>Moder</b> . Man har , udentvivl med Uret , beskyldt Gregor for , at kun Herskesyge og lErgjerrighed var Drioefjeoeren til hans Hand , lemaade , men vistnok bedsmt ham fra et Standpunkt,

<h4>1877</h4>Edwards efterladte Thronfslger , Edward den femte , var kuns tretten Aar gammel . Han befandt sig paa Grcrndsen af Wales med sin <b>Mor</b> , broder . Greven af Rivers . Den gamle Adel , i hvis Spidse stod Hertugen af Buckingham og Lord Hastings , frygtede , at hine forhadte nysop , hsiede af Dronningens Partie nu skulde tilvende sig al Magten , og forestillede Richard af Glouce , sier , at Regentskabet egentlig tilkom ham , samt til , bsde deres Bistand . Richard besluttede at bem < rg « tige sig Thronen for sig selv , og at naae <b>Maalet</b> ved Misgjerning vakte hos ham ikke et Viebliks Betænkning . Han var af en afskrcekkende Legems , hceslighed , og hans <b>Sjel</b> svarede til Legemet . Dog havde han endnu til denne Tid forstaaet at lcrgge noget Skjul paa sin afskyelige Nederdrægtighed ; kun Dronningen synes at have kjendt hans sande Charatteer noget ; men den lumske Richard for , stod at overliste hende . Hun havde befalet sin Broder , Grev Rivers , at bringe den unge Edward til London med sioerk Bedækning ; Richard vidste at overtale hende til at forandre denne Bestemmelse

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016101108104'target=_>Nissen, R. Tønder, 1870, Verdenshistoriens vigtigste Begivenheder i fragmentarisk Fremstilling</a><h4>1080</h4>men da han havde Udsigt til at kunne gjpre det bedre ved at træde i Kirkens Tjeneste , begyndte han at studere Theologi og blev , 22 Aar gammel , viet til Biskop . Men hans Strceben stod til et endnu høiere <b>Maal</b> . I Frankrige regjerede dengang Ludvig 13 , San af Henrik 4. Ludvig var svag og befattede sig ikke synderlig med Regjeringen ; hans <b>Moder</b> , Maria af Medici , havde styret Riget , indtil han blev myndig , og hun og hendes Yndlinger besad ogsaa siden megen Indflydelse . Richelieu vandt Enkedronningens Yndest og Tillid . Hun fik ham ind i Statsraadet , uagtet de celdre Medlemmer sygte at hindre det , da de frygtede hans overlegne <b>Aand</b> . Et halvt Aar senere var han Raadets fvrste Medlem og Frankriges egentlige Regent . Paven gjorde ham til Kardinal . — Under hele sin Statsstyrelse havde han for Me at udvide den kongelige Myndighed og at ydmyge det spansk-vsterrigske Hus > Da han kom til Roret , opfprte de Store sig , som om de ikke vare Undersaatter , og Befalingsmandene i Provinserne , som om de vare uafhængige Herrer . Men Richelieu kuede de Store og befæstede Kongens Myndighed . I dette Memed bekrigede han ogsaa Hugenotterne ; thi de havde sceregne Rettigheder og kunde derfor , naar de vilde , trodse den kongelige Vilje . Han fratog dem deres < s ? ikkerhedspladse , men indrpmmede dem fuldkommen Religionsfrihed. — For at ydmyge Spanien og østerrige blandede han sig i alle Europas politiske Spprgsmaal . Til den Ende understøttede han , som vi senere skulle se , Protestanterne i Tydflland til samme Tid , som han bekrigede dem i Frankrige . Det var naturligt , at Richelieu maatte finde mange Fiender , der arbeidede paa hans Fald . Men alle Sammensværgelser opdagede han itide , og Mange maatte byde med sit Liv for sine mislykkede Anslag . Disse Sammensværgelser vare saa meget farligere , som Kongen i sit Hjerte ikke yndede Kardinalen ; han frygtede ham . - Et af de farligste Anstag truede Richelieu kort wr hans Dyd . Cinqmars ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015012808016'target=_>, 1864, Professor Stenersens Reformationshstorie</a><h4>675</h4>tro og haab paa Gud Fader den almægtige , den rette Ertsflaffer , i hans Sons Navn , der ved sit Ord bragte Sslv og Guld i Fiflens Mund , som endnu stedse lader Ertssen voxe og giver den i rette Tid til dem , som holde ved i deres Gruber , og holde med hjertelig Bon ud hos ham i Taalmodighed . Den evige Gud vil vcere med Eder og paa hans rige Velsignelse og milde Haand «fisser jeg Eder ogsaa fra al Eders Skyld . " Men fsrend denne Mand kom hjem , kom der Budflab til ham paa Veien , at man havde truffet paa god Erts i hans Grube ; Arbeidet gav Udbytte og Manden lsste sig virkelig ud af sin Gjeld . Ved denne Leilighed fortjener og den Tildragelse at omtales , som Katholikerne sagde var Aarsag i hans hastige Dsd . Man forte nemlig en <b>Pige</b> paa 18 Aar til Luther og bad , at han vilde uddrive Djcrvelen af hende ; thi de sagde , at hun var besat . Luther bad hende lcese Troesartiklerne , det gjorde hun ogsaa , men da hnn kom til den anden , tang hun stille og blev gruelig plaget af Djcevelen . Luther sagde da : „ Jeg kjender Dig vel , Dn Djcevel ! Du saa gjerne , at jeg med store Ceremonier vilde besvcerge Dig , men jeg vil ikke gjsre det . " Da Luther Dagen efter prcedikede , blev hun bragt i Kirken og efter Prcrdikenen ind i Sakristiet , hvor hun faldt til Jorden og slog med Hcender og Fsdder . Nogle Studenter reiste hende op igjen , hvorpaa Luther erindrede de Tilstedeværende om , at den Magt at uddrive Djcevle ei lcenger var i Kirken , at den ei heller nu , da Kirken allerede var plantet , lcenger var fornoden , at man ei heller fluide bryde sig om de pavelige Ceremonier , men bede mod denne <b>Aand</b> og ikke scette Gud <b>Maal</b> eller Grcendse , naar han flulde uddrive den , da Gud derved blev fristet : meget mere flulde man fare fort i Nonnen og med Taalmodighed vente den af Gud bestemte Hjcelpctime . Han lagde derpaa Haanden paa <b>Pigens</b> Hoved , stemsagde Fadervor og Troesartiklerne

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014112708017'target=_>Nathusius, Marie, 1873, Ugifte</a><h4>54</h4>temmelig eftergivende mod sine egne smaa Andlingssynder og Hverdagslivets smaa Fristelser . Den Forlegenhed, hvormed hun at undvige Frits ' s Blikke viste tydelig , hvortil hun sigtede . Jeg tror , tilstede Emilie paa sin milde sagtmodige <b>Vis</b> , at Kristendommen ikke blot bestaar i at betjende Herren og at afholde sig fra Verdens onde Vcesen eller i Nidkjcerhed for Guds- HEre ; en Kristens Hovedfiender burde vcrre hans daglige smaa Syndevaner : Troskaben i det Smaa maa ogsaa os fremad i det Store . Jeg ved ikke , hvorfor I faa ofte tale om Nidkjcerhed ? indfaldt Frits , livligt . Fordi Nidkjcerhed , afbrsd hans <b>Moder</b> ham , er en farlig Sag og let hos de Kristne kan vcere en Itring af den samme Hjertets naturlige Tilboielighed , som hos Verdens fremtrceder i Spot og Railleri . Denne Nidkjcerhed for Herrens Sag udscrtter os for samme Fristelse , som Higen efter Verdens Fiendstab til Vevis paa Guds Venskab , og det naturlige Hjertes Selvbedrag er usigelig stort . Men dette kunne Inu Icegge tilside , jeg frygter for , at Emnet er Eder altfor bundløst . Faar jeg blot Lov at gjMe en Vemcrrkning med Hensyn til lykkelige Mgtestaber , sagde Emilie : Maa der ikke altid blive et lykkeligt Wgtestab mellem to virkelige Kristne ? Deri kan du have Ret , svarede Moderen , om end vore Synder stabe mange Sorger og foraarsage mange Kampe under Samlivet , maa dog et M ' gteskab , sluttet i Herrens , altsaa i den rette Kjcerlighedens <b>Aand</b> , blive os til stor Trpst og trofast Hjcelp paa Livets møjsommelige Vei . Gjensidig Kjcrrlighed støtter og hjcelper under Vandringen til et og samme <b>Maal</b> ; et Wgtestab mellem sande Kristne maa altsaa vcere lykkeligt om end i stprre eller

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008071112028'target=_>Jouy, Victor Joseph Étienne de, 1833, Moralen anvendt paa Politiken</a><h4>2015</h4>Qvindernes Lovtalere , blandt hvilke den fsrsie Plads tilkommer den veltalende Thomas , synes paa deres Side at have overskredet <b>Maalet</b> i deres Roes , ved at tillwgge dem en fulokomnere <b>Sjel</b> end vor , og ved at tilskrive deres Dyder A3ren for et Herredsmme , hvortil deres Feil maaskee i ligesaa hsi Grad have bidraget . Blandt os det maa man tilsiaae nyde <b>Qvinderne</b> Forrett tigheder , som Oldtidens Helte ; under Beskyttelse af deres Navn kunne de ustraffede begaae mange Feil .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014102408008'target=_>, 1868, Læsebog for Folkeskolen og Folkehjemmet</a><h4>252</h4>Holland og Frankrige . Paa den Tid var Karl den Enfoldige Konge i Frankrige ; han fandt det bedst at holde Fred med Rolf og bsd ham sit Venskab og et stort Stykke af sit Rige . Rolf gik ind herpaa, tog den kristne Tro og blev den fransk ? Konges Jarl . Efter Skikken der i Landet skulde Rolf nu kysse Kongms Fod ; men det sagde han rentud , at han aldrig vilde gjore . For dog at vare rimelig , bsd han en af sine Hirdmand at kysse Kongens Fod , og denne greb da fat i Foden og lsftcde den op til sin Mund , saa at Kongen faldt vaglangs , og stor Latter opstod . Rolfs larledsmme hed Nord man di , og han og hans Wilinger hed Rudejarlcrne , fordi de boede i Roucn ( Ruang ) , som kaldtes Ruda i vort <b>Maal</b> . Det norske <b>Maal</b> blev ikke lange talt i Nordmandiet ; det var kun faa af Nordmandene , som havde Hustruer med , og , da de siden tog Landets <b>Kvinder</b> tilagtc , blev det franske Sprog Bsrncncs Modersmaal . Men Nordmandenes Sands for Sagavafen og Skaldekunst havde Nordmanncrne arvet , og de var de fsrste , som skrev Sagaer og kvadede i det franske <b>Maal</b> , og dette blev nu snart de Fornemmes Sprog rundt om i Europa og holdtes i stor Agt og Mrc , Nordmannernes <b>Aand</b> og Ridderlighed blev et Monster for hele Europa , og Riddervasenct havde sit egentlige Hjem i Nordmandiet . Siden grundede Nordmannernc et Rige i Neapel og Sicilia , og i Aarct 1066 tog de England , og Kongerne i England ere den Dag idag Gange-Rolfs Mtlingcr .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010012603002'target=_>, 1844, Motiveret Udkast til almind. Lov om Fattigvæsenet i Kjøbstæderne og paa Landet</a><h4>1687</h4>agtvcrrdign Charakteer uagtet , kan frikjende for en <b>vis</b> Grad af religios Sv < rrmerie , er den Forening af barmhjertige Sostre afden protestantiske Troe , der har dannet sigi Rhinegnene, hvilke have oposret deres Liv til at pleie Syge i Hospitaler og i de Elendiges Hytter , lig deres katholske Sosire , for hvem denne Aandsretning imidlertid er lettere at forklare . Jeg saae 11 af disse barmhjertige Sosire i Chariteen i Berlin, hvor de havde udbedet sig at maatte pleie de <b>Qvinder</b> , der lide af de vedersiyggeligsie Sygdomme . De vare tildeels unge <b>Fruentimmer</b> af bedre Familier og robede en ikke almindelig Grad af <b>Dannelse</b> . De saae tilfredse ud med deres Stilling og ytrede dechos ingen Hang til svermerisk Opfatning af Livet og dets <b>Maal</b> .

<h4>2114</h4>Bestemmelsen om i hvilken Menighed en <b>vis</b> Person har Hjemstavnsret forudscrtter en forelobig Afgjorelse af det Sporgsmaal , om han kan betragtes som henhorende til den wlirtembergste Stat . Dette Sporgsmaal afgjores ester Vedkommendes Nndersaatsforhold til Staten . Den , der paa en antagelig trovcerdig Maade kan godtgjore , at han er optagen som wurtembergst Undersaat , eller at hans Fader , eller , hvis han er avlet udenfor HEgteskab , hans <b>Moder</b> , paa den Tid han blev fodt , stod i undersaatligt Forhold til denne Stat , eller til noget af de dermed incorporerede Lande , tilkommer Hjemstavnsret der , indtil han er udtraadt af dette Forhold eller har erhvervet Hjemstavnsret i en anden Stat . Ogsaa uden at siaae i noget af de ovenanfotte Forhold tilstcrdes den Hjemstavnsrtt i Wlirtemderg , der uden at siaae i undersaatligt Forhold til en anden Stat , har havt saarigt selvstcrndigt Ophold i Wilttemberg , eller er fodt der , uden at have havt saarigt selvstcrndigt Ophold i en andrn Stat . ( Lov af Ilte September 1807 , Forfatnings Act . af 25 de September 1819 og Instr . af 22 de Januar 1824 ) Som selvstcrndigt Ophold ansees det ikke , naar Nogen opholder sig for sin Opdragelse eller <b>Uddannelse</b> , eller i privat Tjeneste inden Rigets Gr < rndser eller i en enkelt Menighed af Samme , saasom naar han opholder sig som Discipel i en Skole , som Studerende ved et Universitet , tjener som Lcrrlmg

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016050948139'target=_>Nissen, R. Tønder, 1874, Verdenshistoriens vigtigste Begivenheder i fragmentarisk Fremstilling</a><h4>1159</h4>men da han havde Udsigt til at kunne gjore det bedre ved at træde i Kirkens Tjeneste , begyndte han at studere Theologi og blev , 22 Aar gammel , viet til Biskop . Men hans Strceben stod til et endnu hoiere <b>Maal</b> . I Frankrige regjerede dengang Ludvig 13 , Son af Henrik 4. Ludvig var svag og befattede sig ikke synderlig med Regjeringen ; hans <b>Moder</b> , Maria af Medici , havde styret Riget , indtil han blev myndig , og hun og hendes Indlinge havde ogsaa siden megen Indflydelse . Richelieu vandt Enkedronningens Indest og Tillid . Hun sit ham ind i Statsraadet , nagtet de celdre Medlemmer sogte at hindre det , da de frygtede hans overlegne <b>Aand</b> . Et halvt Aar senere var han Raadets fyrste Medlem og Frankriges egentlige Regent . Paven gjorde ham til Kardinal . — Under hele sin Statsstyrelse havde han for Bie at udvide den kongelige Myndighed og at ydmyge det spansk-osterrigske Hus . Da han tom til Roret , opforte de Store sig , som om de ikte vare Undersaatter , og Befalingsmandene i Provinserne , som om de vare uafhængige Herrer . Men Richelieu kuede de Store og befcestede Kongens Myndighed . I dette Siemed bekrigede han ogsaa Huge not terne ; thi de havde sceregne Rettigheder og kunde derfor , naar de vilde , trodse den kongelige Vilje . Han fratog dem deres Sikkerhedspladse , men indrommede dem fuldkommen Religionsfrihed. — For at ydmyge Spanien og østerrige blandede han sig i alle Europas politiske Sporgsmaal . Til den Ende understottede han , som vi senere stulle se , Protestanterne i Tydstland til samme Tid , som han bekrigede dem i Frankrige .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015072408037'target=_>Nissen, R. Tønder, 1858, Verdenshistoriens vigtigste Begivenheder i fragmentarisk Fremstilling</a><h4>930</h4>Frankrige . Dette skyldtes for en vcesentlig Deel den franske Statsmand Richelieu . Armand Johan du Pies sis . Herre til Richelieu , var yngste Ssn af en fransk Adelsmand. Han var egentlig bestemt for den militcere Stand ; men da hans celore Broder , Viskop af Luyon , gik i Kloster , fik han Haab om at erholde dennes Embede . Han begyndte derfor at studere Thcologi og blev , 22 Aar gammel , viet til Biskop . Men hans Strcebcn stod til et endnu hoiere <b>Maal</b> . I Frankrige regjercde dengang Ludvig kJ , Ssn af Henrik 4. Ludvig var svag og befattede sig ilke synderlig med Regjeringen ; hans <b>Moder</b> , Maria af Medici , havde styret Riget , indtil han blev myndig , og hun og hendes Indlinger besad ogsaa siden megen Indflydelse . Richelieu vandt Enkedronningens Indest og Tillid , og denne forsgedes endnu mere , da det lykkedes ham at udjeevne en Strio , som var opstaaet mellem Enkedronningen og Kongen , og at afvcerge den Borgerkrig , som trnede med at udbryde mellem <b>Moder</b> og Son . Maria troede at stulle finde en fast Ststte i Richelieu , som imidlertid var bleven Kardinal . Hun fik ham derfor ind i Statsraadet . uagtet de crldre Medlemmer ssgte at hindre det , da de frygtede hans overlegne <b>Aand</b> . Et halvt Aar senere var han Raadets forste Medlem og Frantnges egentlige Regent . — Under hele sin Statsstyrelse havde han for Oie at ndvide den kongelige Myndighed og at ydmyge det spansk-ostenigste Huus . Da han kom til Roret , opforte de Store sig , som om de ikke vare Undersaatter , og Befalingsmandene i Provindserne , som om de vare uafhcengige Herrer . Men Richelieu luede de Store og befcestede Kongens Myndighed . I dette Dicmed bekrigede han ogsaa Hugenottcrne ; thi de havde sceregne Fcestningcr , sceregne Hcere og sceregne Rettigheder og kunde dersor , naar de vilde , trodse den kongelige Villie . Han fratog dem deres Sitterhedspladfe , men indrommede dem fuldkommen Religionsfrihed . For at ydmyge Spanien og Dsterrige blandede han sig i alle Europas politiffe Sporgsmaal. Til den Ende understøttede han , som vi senere skulle

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010061506023'target=_>Weenaas, A., 1880, Kristelig Kirkehistorie indtil nyeste Tid</a><h4>1378</h4>hvis Antal nu var naaet op til 150.  De ligge mig , " siger han ,  saa ncer paa Hjerte som Bsrn en kjcer <b>Moder</b> . " Under en heftig Bsrnekuppeeftidemi , som bortrev mellem 2 til 3000 Grsnlcendere , beviste Egede sig som en tru og kjoerlig Fader . Hans Hus stod altid aabent for de Lidende , hvorafder laa Angrebne , Doende og Dode overalt . Hos ham fik de legemlig og <b>aandelig</b> Pleie , og hans <b>Hustru</b> bistod sin Mand med saadan Ndholdenhed , at hun endog derved paadrog sig den Sygdom , der endte hendes Dage . Den 30 te Oktober 1734 gik hun ved en salig Dod hjem til Hvilen. 15 Aars anstrengende Arbeide med Lidelser og Provelser as enhver Art havde svcekket Egedes Helbred , og han bestemte sig derfor til at soge Hvile i Hjemlandet . Den 29 de Juli 1736 holdt han sin Afstedsprcediken over Es . 49 , 4 og drog snart efter til Kjobenhavn , men vedblev ligetil sin Dod at virke for sine kjerre Grsnlcendcrc , fornemmelig ved at arbeide ftaa <b>Uddannelsen</b> as Missioncerer . Egede endte sit bevcegede Liv den ste Novcinber 1758 , 73 Aar gammel , hos sin Svigerson , Prceft i Stubbekjobing ftaa Falster i Danmark . Det as ham paabegyndte Vaerk fortsattes ved hans celdste Son Poul Egede , som furfattede den fsrste gronlandste Grammatik og Ordbog , oversatte efterhaanden Luthers lille Katekismus , Ny Testamentet . Kirkeritualet og Thomas af Kempis ftaa det grsnlandske Sprog og virkcde som prcefteviet Missionoer paa Grsnland indtil 1740. Senere blev han Professor i det grsnlandske Sftrog og dode som titulcer Bisp over Gronland ligesom sin Fader . Dsren var aabnet , senere har Andre kunnet hoste , hvad disse Guds Mcend har saaet . Nu er Grsnlands Iskyst besaaet med Missionsstationer , Kirker og Skoler . Kun paa Dftkysten lever der endnu nogle spredte Hedninger . * )

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009021204071'target=_>Nissen, R. Tønder, 1861, Verdenshistoriens vigtigste Begivenheder i fragmentarisk Fremstilling</a><h4>924</h4>Da Spåmens Magt dalede , hcrvede sig i dets Sted Frankrige . Dktte skyldtes for en vcesentlig Deel den franste Statsmand Rickelieu . Herre til Richelieu , var yngste Son af en fransk Adelsmand. Han var egentlig bestemt for den militære Stand ; men da hans crldre Broder , Biskop af Luyon , gik i Kloster , sik han Haab om at erholde dennes Embede . Han begyndte derfor at studere Theologi og blev , 22 Aar gammel , viet til Biskop . Men hans Strcrben stod til et endnu hoiere <b>Maal</b> . I Frankrige regjerede dengang Ludvia 13. Son af Henrik 4. Ludvig var svag og befattede sig ikke synderlig med Regjeringen ; hans <b>Moder</b> , Maria a f Med ici , havde styret Riget , indtil han blev myndig , og ^ hun ^ vg bendes Indlinger besad ogsaa siden megen Indflydelse . Richelieu vandt Enkedronningens Indest og Tillid . Hun sik ham ind i Statsraadet , uagtet de celdre Medlemmer sogte at hindre det , da de frygtede hans overlegne <b>Aand</b> . Et halvt Aar senere var han Raadets fsrste Medlem og Frankriges egentlige Regent . Paven gjorde ham til Kardinal. — Under hele sin Statsstyrelse havde han for Die at udvide den kongelige Myndighed og at ydmyge det spanskosterrigske Huus . Da han kom ti ! Roret , opfsrte de Store sig , som om de ikke vare Undersaatter , og Befalingsmcendene i Provindserne , som om de vare uafhcrngige Herrer . Men Richelieu kuede de Store og befcestede Kongens Myndighed. I dette Diemed bekrigede han ogsaa Hugenotterne ;

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008091603035'target=_>Becker, Karl Friedrich, 1845, Karl Friedrich Becker's Verdenshistorie</a><h4>1145</h4>Grundideer ; thi de plumpe Magthcwere , som ved de vidcilsiabelige Ansiuelsers Medvirkning vare blevne satte i Besiddelse as heres Magt , bemcerkede til deres Fortrydelse , at <b>Dannelse</b> og Vidensiab indeholde Medgiften til deres egne Vildfarelser . Deraf reiste sig deres Had til al Videnskabelighed og til alle de Anstalter og Stiftelser , som vare oprettede for videnstabelige Biemed . Ikke blot Pariser - Universitetet med dets Faculteter, men ogsaa alle Academier og lcerde Selstaber bleve hcevede , og de botaniske Haver , Cabinetterne , Museet og de offentlige Biblioiheker Mede under Opsigt af en Underviisningscomite , i hvilken He bert , C hau mette og Anacharsis Cloots forestillede Hovedpersonerne . Chabot udtalede dette Parties <b>Aand</b> , idet han erklcerede sig inwd en foreslaaet Underviisningsplan af . den Grund , at dell ledede til Lcerdommens Aristokratie , og Republiken blot behovede Sansculotternes Demokratie . Derimod ftgte Bazire at redde Philosophien og Vidensiaberne ved at <b>vise</b> , hvilke Fortjenester de havde havt af Nevolutionen . Tilsidst tog man den Beslutning , at Undcrviisningscomiteen skulde sorge for Folkcdannelsen ved et Antal moralste Forstrifter og daglig lade disse paa trykte Blade opslaae paa Gadehjornerne . Kongernes foregivne Tyranni havde aabnet Vidensiaberne og Kunsterne umaadeligc Hjclpekilder , og Kirken havde allerede i de Tioer , man kaldte de morke , baseret Folkets Underviisning pcia religiøse Indretningers Grnndlag . Det mecst oplyste Aarhundredes Viisdom omstyrtede med ett Slag alle Hjelpemidler for den hoiere <b>Aandsdannelse</b> og overleverede sine egne Bekiendere og Mestere til Morderoren .  leg har , — siger et Menvidne til Revolutionen * ) — feet Eder gaae til Grunde , Condorcet , Chamfort , Florian , Vicq d ' Azyr , Mcrnd , hvis Navne ville forblive dyrebare for alle Videnskabernes Venner . Chamfort , en af Revolutionens Apostler , befricde sig ved en frivillig Dod fra det forhadte Syn af den . Florian dode i Fcengsel , Vicq d ' Azyr i et Anfald af Vanvid . Ogsaa I bleve offrede , Vailly , Dietrich , Liilguet , Barnave , Lavoisier , Roucher , Andreas Chenier , I bleve offrede , fordi I ikke troede paa Marat . " Lovoisier bad om en Frist af 14 Dage , for endnu at kunne fuldende en chemist Undersogelse ; han fik det Svar , at man ikke behovede Chemikerc , og blev sendt paa Skafottet . Roucher , Forfatteren til Digtet , Maanedernc , " sysselsatte sig i Fcrngslet med en < if sine Sonners Opdragelse . Da han havde faaet sin Dom , gav han ham , da han sendte ham hjem , sit Portratt , som en i Conciergeriet vcrrende Maler havde malet , og som han paalagde ham at levere til sin <b>Moder</b> ; neden under havde han strevet disse smukke Vers :

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009020900018'target=_>Nissen, R. Tønder, 1876, Verdenshistoriens vigtigste Begivenheder i fragmentarisk Fremstilling</a><h4>1073</h4>men da han havde Udsigt til at kunne gjere det bedre ved at trcede i Kirkens Tjeneste , begyndte han at studere Theologi og blev , 22 Aar gammel , viet til Biskop . Men hans Strceben stod til et endnu hsiere <b>Maal</b> . I Frankrige regjerede deng ang Ludvig 13 , Spn af Henrik 4. Ludvig var svag og befattede sig ikke synderlig med Negjeringen ; hans <b>Moder</b> , Maria af Med ici , havde styret Riget , indtil han blev myndig , og hun og hendes Indlinge havde ogsaa siden megen Indflydelse . Richelieu vandt Enkedronningens Indest og Tillid . Hun fik ham ind i Statsraadet , uagtet de celdre Medlemmer sogte at hindre det , da de frygtede hans overlegne <b>Aand</b> . Et halvt Aar senere var han Raadets fyrste Medlem og Frankriges egentlige Regent . Paven gjorde ham til Kardinal . — Under hele sin Statsstyrelse havde han for Bie at udvide > den kongelige Myndighed og at ydmyge det spansk-ssterrigske Hus . Da han kom til Roret , opfprte de Store sig , som om de ikke vare Undersaatter , og Befalingsmcendene i Provinserne , som om de vare uafhcengige Herrer . Men Richelieu kvede de Store og befcestede Kongens Myndighed . I dette Memed bekrigede han ogsaa Hugenott erne ; thi de havde sceregne Rettigheder og kunde derfor , naar de vilde , trodse den kongelige Vilje . Han fratog dem deres Sikkerhedspladse , men indrsmmede dem fuldkommen Religionsfrihed. — For at ydmyge Spanien og Bsterrige blandede han sig i alle Europas politiske Sporgsmaal . Til den Ende understpttede han , som vi senere skulle se , Protestanterne i Tydstland til samme Tid , som han bekrigede dem i Frankrige .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016051948127'target=_>Farrar, Frederic W., 1880, Jesu Liv</a><h4>553</h4>Hendes Tro var stcerk , og hendes Bevæggrunde vare rene , om man end maa boere lidt over med dette cegte <b>kvindelige</b> Trcek af <b>moderlig</b> Loengsel , der bringer enhver <b>Moder</b> til at Mste , at hendes SM bliver cerct i hendes Ncerværelse . Og Marias Ssns Time var da ogsaa nu kommen saa noer , at det maatte anses nødvendigt for ham en Gang for alle at <b>vise</b> sin <b>Moder</b> , at han herefter ikke saa meget var lesns , Marias SM , som Christns , Guds SM ; det var nødvendigt at <b>vise</b> hende , at naar det gjaldt hans store Gjerning , naar det gjaldt hans evige Boesen , da havde disse ham saa dyrebare jordiske Baand tabt al deres Betydning . Han maatte , det koste hvad det vilde , lære baade hende og sig selv , at hans Tanker vare ikke hendes Tanker , og hans Veje ikke hendes Veje . Og denne <b>Lærdom</b> knnde han ikke meddele hende paa en mere bestemt og tillige mere kjoerlighedsfnld Maade .

<h4>864</h4>Tales rige Indhold . Hans Ord stFd lige listige og lige kjerlige til den ensomme Tilhorer som til den lyttende Menneskemængde ; og nogle af sit Riges herligste Aabenbaringer meddelte han ikke til de megtige i Israel , ej heller til de store Folkeforsamlinger , men næsten hellere til den førsagte Raadsherre , der kom til ham om Natten , og til den ringe <b>Kvinde</b> , der talede med ham i Middagssolens Hede ved Jakobs BrMd i Samaria . Og hvad Jesus talede , det angik ikke Tiender og Aftvætninger , det rMte ved Menneskenes <b>Sjel</b> , hans evige Bestemmelse og hans Liv , det rMe ved Troen , ved Haabet og ved Kærligheden . Denne Taler tog sin Text ud af Menneskehjertets inderste Dyb , og hans Tale var saa ligefrem , at enhver kunde forståa den . Han førsmaciede Veltalenhedens Kunstgreb og Skolernes <b>Visdom</b> . Men hans Ord trcengte dog igjennem Marv og Ben og dMte over Menneskenes Tanker . Og for med Billeder , laante fra almindelige Forhold , at belyse og anskueliggøre de Lerdomme for det acmdelige Liv , fom han meddelte , talede han om de gwnne Marker , om Vaarens Blomster og om Treernes Knopper , om den rFde og truende Himmel , om Vind og Regn , om Nat og Storm , om og Lys , om StrM og Flod , om Stjerner og Lamper , om Ild og Salt , om Nkrnd , som brendes , om sønderrevne Kleder og Vinbeholdere , om Perler og Penge , om Net og Fiste , Vin og Hvede , Korn og Olie , Husfogder og Vingaardsmend , Arbejdere og Husbonder, Konger og Hyrder , Hofmend i Linkleder og Brno en i sin Bryllupskledning — ja , alt dette forekommer i hans Taler . Han kjendte Menneskelivet ; han

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009102704114'target=_>Bang, A. Chr., 1878, Om Kristi Opstandelses historiske Virkelighed</a><h4>440</h4>ad Erfaringens Vei til <b>Kundskab</b> om , at , hvad der havde mødt ham , var af dennesidig Virkelighed ( Act . 12 , 7. 9. 11 : Engelen har løst Peders Lænker ) . Hvad nu Angelopahnierne ved Herrens Grav angaar , da fremstilles samme af Evangelisterne ikke som opdpara , men som historisk Virkelighed ; især er det af Vigtighed at lægge Mærke til , at Evangelisten Lukas , der i Acta maaske klarere og skarpere end overhovedet nogen bibelsk Forfatter har distingveret mellem » Syn « og historisk Virkelighed , end ikke med et Ord antyder , at de Engleaabenbarelser , om hvilke vi her taler , skulde være noget andet end af dennesidig Realitet . Dette er vel saa omtrent alle ogsaa enige i , der holder fast paa det bibelske Visionsbegreb . For dem , der i Angelophanierne ved Graven kun ser en Vision , falder dette Begreb i Virkeligheden sammen med Hallucination , Feilsandsning , Produktet af abnorme Funktioner i det menneskelige Nerveliv . Det lader sig imidlertid bevise og det til fuld Evidents , at vore Angelophanier ikke kan være begrundet i noget Saadant . Her er jo ikke Tale om Personer , der paa Middelalderens Asketers <b>Vis</b> har ødelagt sin Helbred med Vaagen , Fasten , Pidskeslag og andre Bodsøvelser, eller som paa Mystikernes <b>Vis</b> under aarelange Kontemplationer har tabt Evnen til at sondre mellem indre Rørelser og ydre Virkelighed ; <b>Kvindernes</b> Sjele er aabenbarlig i fuld normal Tilstand . Hvad disse og det paa én Gang og paa samme Sted iagttager med sine Sandser , det maa tillægges objektiv Realitet ; Hallucination hos dem alle paa én Gang og af samme Indhold vilde være en psychologisk Umulighed 1 ) .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010021803048'target=_>Petersen, N.M., 1854, Danmarks historie i hedenold</a><h4>424</h4>besiddet . Saaledes henrandt andre tre Aar , i hvilke der var udvortes Venstab imellem Hagen Jarl og Gunhilde , men begge oftlagde verelvis Svig mod hinanden . Hagen Jarl , en ivrig Tilbeder af de gamle finske Guddomme , crrgjerrig og underfundig , vilde baade besijerme de gamle Guders Altere og forjage Gunhildes Sonner ; han rejste til Oplandene , og holdt i Hedemarken et Mode med Tryggve Olafsen , Gudrod Bjornsen og endnu stere Hovdinger i det sydostlige Norge ; imellem dem alle stuttedes Venssab og Forbund . Nappe erfarede Gunhilde og hendes Sonner dette , forend de oftlagde Raad om at splitte Forbundet ad , og at bemcrgtige sig de Forbundnes Lande ; ved List skulde det hele Foretagende skjules for den endnu listigere Hagen . Kong Harald og hans Broder Gudrod lode Foraaret ester bekjendtgjore , at de vilde gjore et Vikingetog i Vesterhavet ; Skibene laa fcrrdige , og der holdtes et Drikkelag for man gik ombord . I dette kom man paa Tale om Kongerne , og anstillede , som ej var uscrdvanligt , en Sammenligning imellem begge . Kong Harald rostes som den ypperste af alle Brodrene , men Gudrod sagde i Vrede , at han i ingen Henseende stod tilbage for sin Broder , og det vilde han i Gjerningen <b>vise</b> . Imellem begge Bwdre kom det til en Ordtvist, som endtes med , at de udbode hinanden til Kamp . Hvo kunde da ane , at de paa det forestaaende Tog begge havde i Sinde at virke til eet <b>Maal</b> , og at den ypftede Strid kun tjente til at dcekke dette for andres Djne ? Om Morgenen efter gik de ombord , hver paa sin Flaade ; Gudrod sejlede langsmed Kysten osterpaa , Harald derimod styrede ud paa aaben So , for , som det hed , at drage i Vesterviking. Gudrod kom med sin Flaade til Folden , og sendte Bud til Tryggve Olafstn om denne vilde mode ham , for at de kunde aftale et Tog til Landene ved Bstersoen . Tryggve , der erfarede , at Godrod kun havde faa Folk med sig , lovede at komme ; hans <b>Kone</b> , Astrid , sogte vel at afholde ham

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111400035'target=_>Funcke, Otto, 1878, Reisebilleder</a><h4>267</h4>Scet nu , at <b>Piger</b> , forn ere faldne en eller flere Gange , virkelig stulde vcere foragteligere og mere bredefulde end andre Syndere , faa ere de dog ganske vist ulykkeligere , mere trcengende til Hjcelp og Redning , og , Sagen feet i kristelig <b>Aand</b> , ogsaa vcerd at hjcelpes og frelses . Alle ere fkabte for Himmeriges Rige , og den , som er lcengst fjernet fra <b>Maalet</b> , bsr ivrigst opsyges . Det forvildede Faa r i Brkenen vcekker den gode Hyrdes segende Kjcerlighed mere end de 99 , som ere blevne hjemme i Folden .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013020524029'target=_>Holberg, Ludvig, 1834, Niels Klims Reise under Jorden</a><h4>182</h4>ledes mm Foragt til Beundring . Denne Rettcrgangsmaadc , som jeg saa übetcenksomt havde opholdt mig over , blev mig ftaledes forklaret : af mit Legems <b>Dannelse</b> havde de slutter , jeg var en Indvaaner af Firmamentet . Jeg havde syntes at ville voldtage en cerbar og dydig <b>Dame</b> af fsrste Rang , og for denne Forbrydelse , hvori al Formodning domte mig fiyldig , blev jeg fsrt paa Raadstuen . Den ene af Sagførerne havde forstsrret min Misgierning , og paastaaet , jeg burde lide den i Loven paalagte Straf ; den anden derimod afbad vel ikke Straffen , men raadte til at opscctte den , indtil man fik at vide , huo og hvorfra jeg var , og om jeg var et umeelende Dyr , eller et fornuftigt Boesen . Grenenes Opleftelsc mod Himlen , fortalte man mig , var en scrdvanlig Religions-Ceremonie , som man altid iagttog , for man dsmle i Sager . Sagforerne vare omsvobte med Faarestind , for at erindre dem om den Uffyloighed og Oprigtighed , de burde <b>vise</b> i deres Sagers Behandling . Og virkelige « lagde jeg Mcerke til , at alle Sagfsrcre her vare meget retffafne og ustraffelige , hvilket beviser , at det dog er muligt , i en velindrettet Stat at finde redelige og brave Aduocater . Man har saa strenge Love i dette Land mod Sagfordreiere , at ingen Kroglove , Lumskhed eller Partiskhed kan besmykkes . Intet Bedrageri kan afbedes , ingen Misgierning undgaae sin Strafs Det er lige faa umuligt for en Forbryder at undftye , som at finde en Huser eller Hceler .

<h4>729</h4>Den fornemste Aarsag til denne store Ulighed maa soges i Landenes forstiellige Natur , som de mangefarvede Marker , de sorssiellige Jordarter , Planter , Frugter og Urter noksom give tilkiende . Hvad Under , at der ved denne store Forsticllighed i Jordbunden og Frugterne gives saa ulige og hinanden modsatte Egcnstabcr og Scrder hos Indbyggerne? i vor Verden er Forsticllen imellem de la-ngst fra hverandre adspredte Nationers Seeder , Tamkcmaade , Farve , <b>Sjels</b> og Legems <b>Dannelse</b> ikke saa betydelig ; thi da Jordbunden ncestcn overalt er den samme , uden for saa vidt at det ene Land kan vcrre frugtbarere end det andet , og > Vand , Frugter og Urter have ncrsten een Natur , kunde her umuligen frembringes saa ul ' ge Skabninger , som i den underjordiske Verden , hvor hvert Land har fin ganske scrregne Beskaffenhed . Fremmede have vel Frihed til at reise og handle overalt , men ikke til at bosattte sig nogetsteds , hvilket heller ikke Landenes forstiellige og indbyrdes modsatte Vestaffenhedtilladeri derfor cre alle de Fremmede , man <b>moder</b> , enten Vandrende , eller Kiobmamd .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008112403026'target=_>Strauss, Friedrich Adolph, 1861, Sinai og Golgatha</a><h4>112</h4>Imellem Pnyr og Akropolis , adstilt fra dette ved en Dal , er paa en lav Hsi det Sted , fom vi fsrst opssgte i Athen , og som der blev os det vigtigste . Det er Areopagus , hvor Athenienfernes øverste Domstol holdt sine <b>Moder</b> , ligeledes under fri Himmel . Endnu bemerker man de Trin , som fra Dalen , der ligger imellem Areopagus og Pnyx , forte op ad Hsien ; ovenpaa er der hugget Sceder til Dommerne ind i Klippen ; to afsondrede Klippeblokke kunne betegne Klagerens og den Anklagedes Sceder . For Areopagus- Domstolen i ) fsrtes Apostelen Paulus . For Athenieusernes forsamlede Folk , for dets <b>Vise</b> , Epikureerne og Stoikerne , og Alle , som ikke gave sig af med Andet end at hore Nyt , holdt han en Tale , som er bleven et uopnaaeligt Forvillede for alle Talere . Han saae ud over Staden , hen til det i sin Marmorglands straalende Akropolis , til Theseus ' s Tempel og alle de herlige Templer , om hvilke <b>Kundskaben</b> neppe er kommen til os , og sagde :  Gud , som haver gjort Verden og alle Ting , som ere derudi , han , som er Hunlens og Jordens Herre , boer ikke i Templer , gjorte med Hcender . " Han saae Gudindens Billede , som ragede op over Akropolis , tenkte paa Phidias ' s Vcerk og de utallige Gudebilleder , som prydede Staden , og sagde :  Efterdi vi da ere Guds Slcegt , skulle vi ikke mene , at Guddommen er lig Guld eller Sslv eller Steen , formet til et Billede ved Menneskenes Konst og Paafund . " Han forkyndte de athenieusiste Mend , som i alle Maader vare omhyggelige for deres Gudsdyrkelse, den übekjendte Gud , som de ukjendt dyrkede . Dermed udtalte han tillige Dommen over Hedenskabets hoieste Blomstring i Konst og Viisdom , over Menneskeaandens Kraft overhovedet , at den vel kan soge den sande Gud , men ikke finde ham ; han

<h4>1833</h4>paa den Dag skal han borttage Fodringens Smykker , de smaa Sole og de smaa Maaner , Z3renringene og Armkjederne og SWrene , Hovedsmykkerne , Fodljederne og Baandene , Lugtedaaserne og Amuleterne " o . s . v. Men <b>aandelig</b> <b>Dannelse</b> ere <b>Kvinderne</b> ganske fremmede for , og saameget tommere viser derfor deres ndvortes Prydelse sig . De rcekkede os selv Piberne , hvilke de selv i Forveien rsg af ; derpaa blev der budet fortrceffelig Limonade og Kafse . Denne , som stsdes i Mortere , koges med Sukker , og drikkes uden Melk tilligemed Vundfaldet af ganske smaa Kopper , som istedetfor Underkop rcekkes paa Fodder af Metal , ofte af Guld og Sylv , ligesom vore 3 Eggebcegere . <b>Kvinderne</b> ryge i Almindelighed her , hvorfor der trods den billige Tobak daglig skal ryges i denne By for henved tredivetusinde Pjastre eller ncesten tretusinde Nigsdaler . Foruden Narghilen og Tschibuken har Damascus endnu en sceregen Art Vandpiber , ved hvilke Våndet ikke opbevares i Glas- , men i Messingkugler .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015060108051'target=_>Bang, A. Chr., 1876, Om Kristi Opstandelses historiske Virkelighed</a><h4>781</h4>medens han i Acta skulde give tilbedste en anden Tradition, ifølge hvilken Herrens Himmelfart først skede 40 Dage senere . Naar man ikke undser sig for at behandle Evangelisterne paa saadan <b>Vis</b> , da kan man selvfølgelig faa ud , hvad man ønsker . Kun staar man da ikke paa Historiens , men paa de frie Fantasiers Grund . Som bekjendt har man i Beretningerne om den Opstandnes Aabenbarelser troet at finde tre Slags Traditioner , en Jerusalemisk , en Galilæisk og en Jerusalemisk-Galilæisk . Men alle disse Traditioner existerer kun i de negative Kritikeres Hjerner og forresten ikke . Thi Mathæus beretter baade om Christophonier i Jerusalem og Gaiilæa ; det samme er Tilfældet med Johannes , medens Lukas kun fortæller om den Opstandnes Aabenbarelser i Judæa . Markus ( i det utvivlsomt ægte Afsnit 16 , 1 — 8 ) derimod tier baade for Jerusalems og Galilæas Vedkommende . Sidstnævnte Evangelist , forsaavidt som hans Evangelium er bleven fuldført, maa have seet sit <b>Maal</b> naaet ved at fortælle , hvad der tildrog sig ved Graven . At antage , at han , der havde staaet i et saa nært Forhold baade til Paulus og Peder , ikke skulde have kjendt til ialfald flere af den Opstandnes Aabenbarelser , er ligetil en Umulighed . Mathæus har ogsaa anseet Begivenhederne ved Graven og Englenes Forkyndelse af Opstandelsen for sin Fremstillings Hovedformaal ; han er dog gaaet et Par Skridt videre end Markus , idet han som Bevis for , at Herren virkelig er opstanden og altsaa er den forjættede Messias ikke blot beskriver Aabenbarelsen for <b>Kvinderne</b> , men ogsaa den paa hint Bjerg i Gaiilæa ; maaske er han ved 28 , 7 og 10 bleven foranlediget til at meddele , at Herren virkelig traf sammen med sine Disciple paa det omtalte Sted . Lukas har vistnok nærmest af historiske , maaske ogsaa af dogmatiske Hensyn ( cfr . I Kor . 15 ) følt sig opfordret til blandt Øienvidnerne ( cfr . Luk . 1 , 2 ) at skaffe sig Oplysning om flere af

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016053026003'target=_>Monsen, Chr., 1849, Foraarsgave for Damer</a><h4>57</h4>lig et halvgennemsigtigt Flor , paa engang gav deres Tanker en anden Retning og lod dem forglemme, idetmindste for Vieblikket , Fader , <b>Moder</b> og Hjem , kort Alt , som hidtil havde sysselsat deres <b>Aand</b> og givet dem Anledning til at tolke sine Meninger og Folclscr for hinanden . Det var med en Art af LGrbodighed og dyb Fslelse de- betragtede den vidt udbredte Stad , der laae for dem . Det var i den deres aandige Liv stulde udvikle sig , hvori de siulde modtage den hoiere <b>Dannelse</b> , hvorom de endnu ikke havde noget fuldstændigt Begreb , men som de dog tcenkte sig som Noget , ' der var vcerd den hoicste Strceben , og de betragtede Staden , der laae for dem , som man betragter en Lærer , hvis Alder gjor ham cervcerdig og hvis Dygtighed man maa agte . I disse Vieblikke opsteg vistnok det Forscet i begges Sjele , at arbeide troligcn for at naae <b>Maalet</b> ; for deres Tanker fremstillede sig alle de Mænd af Vetydenhed og <b>Lærdom</b> , som herfra vare udgangne, og de folte sine Kinder glode , ikke alene ved det stolte Haab at komme til at ligne dem ; men ved den ungdommelige Glæde , som saa let forer selve Haabet og Beregningen for vidt , saae de sig allerede i Aanden hcedrede som Koryphser i Geniernes og Talenternes Række .

<h4>190</h4>„ Virksomheden har aldrig tomme Hænder " , sagde Uhrmagerw . „ Og , stritte talt , har jeg formegen Kjccrlighed til mit Barn , til at see paa Penge og Stand , naar det gjældcr hendes Lykke . Den vil jeg strcebe at sorge for saa godt jeg kan og jeg er , stritte talt <b>vis</b> paa , at Du , hvad den Ting anbelanger , ikke tcenker anderledes end jeg , men vil arbeide med mig til eet <b>Maal</b> . Jeg har det Haab om Milla , at hun ikke uforsigtig stutter nogcnsomhelst Forbindelse for hnn er overbeviist om , at den hun vcelger til sin Livsledsager ogsaa elsker hende oprigtig . Jeg har endnu ikke sporet Forfængelighed hos hende , og naar denne ikke er tilstede hos <b>Fruentimmerne</b> , saa ere de , stritte talt ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010012603023'target=_>Hanson, Peter Treschow, 1832, De mærkeligste Tildragelser i Verden siden Katharina den Andens tronbestigelse</a><h4>1808</h4>Staten . Vindffibeligheden , siden Krigens Begyndelse indffr < rnket til Troppernes Udrustning , henvendte sig igjen til Ouerdaadigheos-Gjcnstande og Moden svingede igjen dens Scepter , isar da Directeuren Barras ved en glimrende Neprcrsentation ssgte at erstatte Savnet af et Hof . Alr bibeholdt dog et republikansk Prag og Dat mernes Paakladning rettede sig muligst efcer hvad der havde vcrret Skik og Brug i de gamle gra ? ske Fristater . Dagens Dronning var Madame Cabarrus < Tallien . Hun havde stemmer hendes davcrrende Mand til Menneskes lighed og opfiammet hans Mod til at bekcrmpe N < rdselsl mamdene ; nu glcrdedes Kunstnerne ved hendes Smag , Videnssllbsmcrndene ved hendes <b>Kundskaber</b> , Cirklerne hos Varras ved hendes <b>Aand</b> og Folket ved hendes fortrylt lende Skjonhed , naar hun viste sig ved offentlige Lei , ligheder . Senere gjorde den yndige Madame Recamier , en riig Banksrs <b>Kone</b> , hende Rangen noget stridig .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011052520006'target=_>, 1880, Kalila og Dimna</a><h4>315</h4>Dimna sagde : Man fortceller , at der i en <b>vis</b> Stad var en Lcege , som besad <b>Kundskaber</b> og Velvillie , og ved de Kure , han havde udfort , havde erhvervet sig et Navn . Men han var blcven gammel , og hans Syn var svagt . Nu havde Fyrsten i denne Stad en Datter , som han havde givet en Broderson til <b>Hustru</b> . Saadanne Smerter , som pleie at indfinde sig hos Svangre , indfandt fig nu ogsaa hos hende . Man lod hiin Lcrge kalde , og da han kom , spurgte han den unge <b>Kone</b> om hendes Ildebefindende . Hun underrettede ham derom , og han forstod , hvad det var for en Sygdom , og hvilke Lcegemidler derved udfordredes, og han sagde : ~ Dersom jeg blot saae , saa skulde jeg ved mit Kjendskab samle de mangehaande Lcegemidler , og jeg forlader mig heri ikke paa nogen uden mig selv . " Nu var der i Staden en enfoldig Mand ; han fik Nys om Sågen . Han gik til dem og gav sig ud for Lcege , ligesom han ogsaa lod dem vide , at han godt forstod at blande Lcegemidler og Urter , at han kjendte saavel de sammensatte som de enkelte Lcegemidlers Egenskaber . Fyrsten befalede ham at gaae ind i Apotheket og tåge de Lee-

<h4>464</h4>at ydmyge sig for den , som er ringere end han , indtil han har naaet sit <b>Maal</b> ; da fryder han sig over Vegivenhedernes Udvikling og Fslgerne as sin Taalmodighed . " — Kongen sagde : ~ Fortcel mig nu om de Kloge blandt Uglerne." — Ravnen sagde : ~ leg fandt ingen Kloge , undtagen den , som holdt for , at de skulde drcebc mig , og flere Gange sogte at ophidse dem . Men de vare overmaade ukloge , anstillede ingen Betragtninger over min Stilling , men sagde , at jeg var en Stormand blandt Ravnene , og jeg blev mere agtet end de Kloge . De vare ikke paa sin Post imod min List og mine Rcenker ; de gave ilte den Raadgiver Gehsr , som forstod dem , og de lagde ikke Skjul paa sine Hemmeligheder . De <b>Vise</b> have sagt : ~ Kona , en bor skjule sine Anliggender for Dretudere og ikke lade uogen gjennemstue sive He : umcligheder . " Kongen sagde : ~ lntet har fordcervet Nglerne , synes mig , udcu Uretfcerdighed og Kongens svage Forstand , saavclsom hans Eftergivenhed for flette Ministre . " Ravnen sagde : ~ Du har Ret , o Konge ; sjelden bliver nogen rig uden at blive overmodig ; sjelden fatter en Maud Tilboielighed til <b>Qvinden</b> , nden at det kommer nd ; sjclden spiser uogen formcget ndeu at blive syg , og sjelden stoler en Konge paa slette Ministre uden at styrte i Fordcervelse . Man har sagt : ~ Den ZEdle finder itke Behag i stidue Lovtaler , Bedrageren ikke i mange Venner, den Sletopdragne ikke i Wdelmod , den Gjerrige ikke i Veldcrdighcd , den Graadige ikke i smaae Forseelser , og den overmodige Kouge , som forsommer sine Anliggender og har slette Miuistre , ikke i Rigets Fasthed og Undcrsaatternes Vel . " Kougen sagde : ~ Du har dog lidt meget Ondt ved at forstille dig og ydmyge dig for Uglerue , Navncn sagde : ~ Den , som lider Oudt i Haab om Fordele , giver al Skam og Uudseelse eu god Dag og vceuner sig til Taalmod , Folgerne af haus Klogskab blive roste , ligesom Slaugen ved Taalmodighed fandt sig i at bcere Frosternes Konge

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013020424026'target=_>Holberg, Ludvig, 1880, Niels Klims underjordiske Reise</a><h4>164</h4>dette troede jeg ogsaa var alt , hvad man kunde sige til deres Berømmelse . Mens jeg saaledes i Stilhed smaasnakkede med mig selv , kom et Trcee ind ad Doren med en trekantet Tingest i Haanden , hvormed han , efterat have lost op for mit Bryst og blottet min eene Arm , meget net slog Hul paa Median-Aaren . Da han havde taget saa meget Blod , som ham syntes nok , forbandt han Armen med ikke mindre Fcerdighed , besaae med megen Opmærksomhed Blodet , og gik med Kiendetegn paa en Slags Forundring stiltiende bort . Alt dette styrkede mig endnu meere i den Mening , jeg havde fattet om dette Folks Taabelighed. Men da jeg siden efter lcerte det underjordiske Sprog , og sik et tydeligt Begreb om alle disse Ting , forvandledes min Foragt til Beundring . Denne Rettergangsmaade , som jeg saa übetænksom havde opholdt mig over , blev mig saaledes forklaret . Af mit Legems <b>Dannelse</b> havde de sluttet jeg var en Indvaaner af Firmamentet . Jeg havde syntes at ville voldtage en cerbar og dydig <b>Dame</b> af forste Rang , og for denne Forbrydelse , hvori al Formodning domte mig skyldig , blev jeg fort paa Naadstuen. Den eene af Sagforerne havde forstorret min Misgierning , og paastaaet , jeg burde lide den i Loven paalagte Straf ; den anden derimod afbad vel ikke Straffen , men raadte til at ovfcette den , indtil man sik at vide , hvem og hvorfra jeg var , og om jeg var et umcelende Dyr eller et fornuftigt Vcesen . Grenenes Oploftelse mod Himmelen , fortalte man mig , var en scrdvanlig Religions-Ceremonie, som man altid iagttog , for man domte i Sager . Sagforerne vare omsvobte med Faarestind , for at erindre dem den Ufiyldighed og Oprigtighed de burde <b>vise</b> i deres Sagers Behandling . Og virkelige « lagde jeg Mcerke til , at alle Sagforere her vare meget retskafne og ustraffelige , viser , at det dog er mueligt i en vel indrettet Stat

<h4>189</h4>bragte det meget videre i <b>Lcerdom</b> ved stille Lcesning og rolig Overtcenken . Dette Svar , skiondt det syntes artigt nok , tilfredsstillede mig alligevel slet ikke . I denne Bye var for Resten et Gymnasium eller Academie , hvori de boglige Konster lcertes med megen Alvorlighed og Sommelighed . Min Vert forte mig op paa denne Hoistoles Auditorium en <b>vis</b> hoitidelig Dag , da der blev creeret en Madik , eller Lcerer i Philosophien . Denne Handling gik for sig uden mindste Ceremonie , undtagen for saavidt at Candidaten holdt en net og grundig Tale over en <b>vis</b> physist Opgave , hvorpaa hanafAcademiets Forstandere blev indsirevet i deres Tal , der have Tilladelse til at loere offentlig . Da min Vert spurgte mig , efterat vi vare komne hjem , hvorledes denne Promotion anstod mig , svarte jeg ham , at den forekom mig alt for tor og maver , i Forhold til vores . Jeg fortalte ham , at Magistrene og Doktorerne hos os maatte disputere for de bleve creerede , ved hvilken Efterretning han trak forskrækkelig paa Ncesen . Han spurgte mig med en Slags Spodsthed , om disse Disputatsers Beskaffenhed , hvad Maade de bleve holdte paa , og hvori de afvege fra de hos dem brugelige . Jeg fagde ham , at de handlede i Almindelighed om lcerde og vigtige Materier , isoer saadanne , som angik Scederne , Sproget og Klædedragten hos to gamle Nationer , der i forrige Tider havde fornemmelig blomstret i Europa . Jeg forsikkrede ham , at jeg selv i tre meget lcerde Disputatser havde sagt alt hvad der kunde siges om disse to gamle Nationers Tofler . Ved at hore dette faldt han i Latter , og skoggerloe saa hoit , at det gierlod i heele Huset . Hans <b>Kone</b> , som Horte denne Latter , kom lobende af alle Krcefter ind til os , nysgierrig efter at vide hvad der havde foraarsaget den ; men jeg var bleven saa opbragt derover, at jeg ikke vcerdigede at svare hende ; thi hvo kan udholde at hore saa vigtige Ting optages med

<h4>241</h4>Indbyggere , som man iscer bemcerker paa Landet i Bondestanden , hvor alle ere indfodte ; thi i de store Byer , og iscer i Hovedstaden , er en Sammenblanding af alle Slags Trceer . Den Formodning , jeg havde giort mig om disse Potuaneres Klogskab , bestyrkedes meer og meer , jo notere jeg lcerte at kiende deres gode Egenskaber . De Love og Skikke , som meest havde Ddt mig , fik efterhaanden , som jeg indsaae deres Billighed og Retfcerdighed , mit fuldkomneste Bifald , og min ugrundede Foragt forvandledes til gruudet Beundring . Det vilde vcere mig en let Sag een for een at opregne de Ting og Scedvaner , som jeg ved lofelig Stuen havde holdt for daarlige; men ved Wiere Betragtning fandt grundige og viise . Af tusinde vil jeg blot anfsre eet eeneste Exempel , der scerdeles levende skildrer dette Folks Caracter . En <b>vis</b> lcerd Philolog ansøgte om Rectoratet i et af Fyrstendømmets offentlige Skoler . Hans Ansøgning var belagt med en Anbefaling fra Borgerne i Byen Nahami , der bevidnede , at Candidaten i samfulde sire Aar havde levet fredelige « med en lMagtig og utroe <b>Kone</b> , og i al den Tid baaret sine Horn med Taalmodighed . Vidnesbyrdet lDd omtrent saaledes : Saasom ncervcerende hcederlige og vellcerde Mand loktan Hu , har af os vceret et Vidnesbyrd om sit Levnet og Opførsel begierende , bevidne vi underskrevne Vorgere og Boesiddere i den Afdeling af Byen , fom kaldes Posto , at samme Mand har i sire samfulde Aar levet i fredelig Stilhed med en utroe <b>Kone</b> , baaret sine Horn med Taalmodighed, og viist saa megen Sagtmodighed i at taale dette Onde , at vi holde ham fuldkommen vcerdig til det ledige Rectorat , dersom i øvrigt hans <b>Lcerdom</b> svarer til hans Opførsel . Nahami , den tiende i Palme-Maaneden , Aar Tretusinde efter den store Oversvømmelse . Til dette Vidnesbyrd var foiet et Testimonium

<h4>620</h4>den fineste Levemaade . Men jeg havde heller seet , at hun havde vceret lidt mindre artig ; thi det var mig ikke synderlig behagelig at rcekkes Mad af en Soes Hcender . Man maa lcegge Mcerke til , at Indbyggerne i KeiserdMmet Mezendore , skiondt de i Legems <b>Dannelse</b> ganske ligne Dyrene , dog have Hamder og Fingre paa Forbeenene . Da de have enten Haar , Uld eller Ficer paa Kroppen , behøve de ingen Klceder . Imidlertid udmcerke dog de Rige sig fra de Fattige , ved visse Prydelser , som , for Exempel , Halskieder af Guld eller Perler , og Baand , fom de i Alental vinde omkring deres Horn . Commandeurens Frue havde faa mange Baand og Slmfer paa Hovedet , at man knap kunde skimte hendes Horn . Hun giorde Undskyldninger for hendes Mands Fraværelse , og fortalte , at han maatte blive hiemme , formedelst en Trcette han nylig var bleven indviklet i , og holde Raad med to Skader , som den fplgende Dag stulde ftre hans Sag for Retten . Efter Maaltioet havde den spraglede Soe , Renovations-Infpecteurens <b>Kone</b> , en Samtale under fire Bine med vor Tolk , hvori hun aabenbarede ham , at hun var dødelig forelstet i mig . Han trøstede hende det bedste han kunde , lovede hende sin Understøttelse , og vendte sig til mig , for at overtale mig ; men da han mcerkede , at al hans Veltalenhed var spildt , raadte han mig , at tage Flugten , da han vidste , at denne <b>Dame</b> vilde scette baade Himmel og Jord i Bevcegelse for at opnaae sit Onste . Fra den Tid af blev jeg bestandig ombord , ister da jeg kom efter , at en gammel Galan hos denne Frue , en <b>vis</b> Btu6ioßUß ? KiloßopKi3L , af Skinsyge stod mig efter Livet . Men Skibet selv var mig ikke Vcern nok mod denne Dames Anfald , som snart ved Mellembud, snart ved Kærlighedsbreve , og snart ved Elskovs Elegier , strcebte at smelte den lis , som omgav mit Hierte . Havde jeg ikke . ved det Skibbrud , jeg siden leed , mistet disse Papirer , skulde jeg have fremviift

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2015060948107'target=_>, 1845, Udvalg af Corsarens to første Aargange</a><h4>1570</h4>Forst maae vi dog fortoelle , at Forfatteren ikke har villet vente paa den Opfordring , som efter et eller fiere Mesterværkers Udgivelse pleier at gjsres til Digterne om , selv at fortcelle deres Liv og Levnet . Han har med den siorste Forekommenhed og Artighed alt forud opfyldt denne Fordring , som han nok indsaae , ikke vilde udeblive , og berettede desaarsag , for at <b>vise</b> , hvilken Natur ( nemlig Digternatur ) , der alt tidlig stak i ham , at han aMrede i sit 3 die , 4 de Aar blev fortryllet af „ <b>kvindelig</b> Formstjsnhed", * ) og at han i sit ste Aar grced over „Mindeblomster paa Abrahamsens Grav " . Ja , , han viste alt tidligt den fslsomme Sjcel , der skulde udkloekke „Kjeerlighedens Due " . ( Som et mcerkeligt Exempel paa Forfatterens Svulst stulle vi gjsre opmcerksom paa den Maade , hvorpaa han ved denne Leilighed omtaler sin afdsde Moders Udseende : „ Hendes Ncese indtog det bestemte <b>Maal</b> efter Ansigtspartiernes kunstneriske Inddeling". Formodentlig gjsr Erindringen om den kjcere

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011062923013'target=_>, 1880, Samling af religiøse breve skrevne af forskjellige forfattere i det nittende aarbundrede</a><h4>223</h4>det celdste 12 , det yngste V 2 Aar gammelt og er Pattebarn . Det er steet af Herren og underligt for vore Dine . Han var fodt i Juli 1809. Han kunde endnu havs arbeidet lcenge i Herrens Vinaaard og udstroet megen Sced til Guds Navns 3 Sre . Men det var ikke saa Herrens Villie , han fcmdt det bedst at sende ham hjem , og lade hans Virksomhed , hans Strid og Prover vcere endt . Kinn Prcestegjceld er nu , med Hensyn til Lcegmcmds virksomhed , som Faar uden Hyrde . Ingen som kan lede Missionsmoder og Opbyggelser ; ja det ser saare fattigt ud . Men han som er kjendt i luda og har et stort Navn i Israel , som har sin Hytte i Salem og sin Bolig i Zion , ja han som kan opvcekke Abraham Born af disse Stene , han kan ogsaa give en Abrahams Son i hans Sted . Den stcMels blinde Stefansen , som er Sogneprcest i Kinn , er en meget daarlig Veiviser paa Sandhedens Vei , og det vcerste er : Han vil vcere alle Venners Ven , og har i Grunden et fiendtlig ! Had til al Lcegmcmdsvirksomhed . Naar han cndda ikke bar Kappen paa begge Skuldre , var det mere tilgiveligt . Herren aabue hans Vme , baade for sig selv og hans Menighed, at han maatte faa det rette Syn paa Sagen og komme i alvorlig ! Arbeide medens det er lys Dag . Hvorledes tever din kjcere <b>Kone</b> ? Hun er kjcerlig hilset ; fra hendes Tante Guro Tingnces kan jeg hilse hende . Hun er over al Forventning frist og livlig ; hendes meste Arbeide er at stelle med sin Sonnekone og hjcelpe hende til med de mange smaa Born , i et og alt som behoves og tiltrcenges , desuden staar det godt til paa Tingnces , hvad Sundhed og Enighed cmgaar , og for det indre arbeider de unge og den gamle <b>Moder</b> til et <b>Maal</b> . Det gaar ellers med det samme gamle her paa Befring . Af Ordets Horere er her nok mange , men af dets Gjorere er her saare faa , imod de mange Gud formene sit Rige ved sit Ord og <b>Aand</b> , og for alle Ting giv os , forn endnu er opholdt og bevaret i Sandheden , Naade at trcenge os mere og mere frem igjen-

<h4>251</h4>Inderlig elstet og ofte ihukommet Broder i Kristo med <b>Kone</b> og Born og ovrige Venner , som vil elske den Herre lesum . Eder tilonstes Naade til Bestcmdighed og Fremgang paa den gode Vei , som er os cmbefalet i Ordet , og ved den Helligacmds Vidnesbyrd beseglet i vore Hjerter , at saafremt vi blive i denne erkjendte Scmdhed , efter denne gode Regel Ordets übedragelige Rettesnor , der viser os hen paa ham som er Veien , Sandheden og Livet , ja tillige det Lys , som er kommen i Verden for at oplyse alle som vil lade sig oplyse , lede , lcere og veilede . Disse giver han og af sin <b>Aand</b> , og de , som drives af ham , disse ere Guds Born ; thi de ere fodte af Gud og tragte efter at gjore hans Villie : thi ligesom de erkjende , at dem er meget forladt, saa ere de og skyldige at elske meget . O ja . Venner , nåar vi betcenke den store Guds Kjcerlighed imod os , som er bevist derved iscer , at han sendte sin Son den Enbaarne og det til os , som vare hans Fiender og derved have faaet Naade og Magt formedelst Troen paa dette velsignede Frelsernavn til at vorde hans Born , udvalgte formedelst en evigfrelsende Raadstutning i det treenige Raad og delagtiggjort i Forjcettelsens salige Naadepagt til Eiendomsfolkets herlige Forlosning , og om vi end vare som Fremmede, ere vi dog forte ncer til , og nu formedelst Haabets herlige Stjerne kan fryde og glcede os med en usigelig og herliggjort Glcede , idet vi vide , at ester en kort og stcMet Provelsestid stal Kronen vindes , og straale paa vore Hoveder i Salighedens Hjem . Og nåar vi nu da ved at alt dette , ja al den Naade , vi her har og nyder , og hist stal faa i langt fuldere <b>Maal</b> , nåar det fulokomue aabenbares , nåar Zions forloste stal vende hjem med Frydesang , at alt dette , siger jeg , har vi ene og alene ham at takke for , forn traadte Persekarret for os alene , da mener jeg , vi og har storste Aarsag til at takke og prise ham , som os kjobte af Jorden med sit Blod . Derfor , dyrekjobte , soger vi at <b>vise</b> os som det sommer sig hans Eiendoms Folk , og i vort

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013100708169'target=_>, 1879, Fra Sømandsmissionen</a><h4>705</h4>Kampe , for menneskelige Forhaabninger og for menneskelige Skuffelser . Under alt dette er Herrens Ord blevet det samme . Det forkynder den samme evige Magt , ved hvilken alle Ting ere blevne , ved hvilken det opholdes , den samme hellige Lov , som Mennesker vel kunne trodse , men ikke forandre , efter hvilken Ret altid bliver Ret og Uret altid Uret , den samme forbarmende Kjerlighed , der vel kan antages og forkastes , men fom uforandret tilbyder sig som den eneste Vei til Frelse og til Salighed. Men dette samme i sit Boesen saa uforanderlige og urokkelige Guds Ord er paa samme Tid saa omhyggelig for ethvert Menueskes Fornsdenheo og Evne som en <b>Moder</b> , der nsie agter paa , hvad den Spede i hendes Skjod kan bcere . Det ikleder sig alle Folkeslags Tnngemaal. Det taler uudgrundelig <b>Visdom</b> til de <b>Vise</b> , det gjor sig enfoldig for de Enfoldige . Det stiller Fuldkommenhedens hoie Fordring til de Retfærdige , og det behandler de Svage som det brudte Ror , om hvilket der stal opbindes , som den rygende Tande , i hvilken der stal pustes Flamme . Det kalder Bsrnene til sig for at fortcelle dem , at Guds Rige horer dem til , og det taler venligt til de Gamle og Modige for at indbyde dem til at ssge Hvilen for deres Sjcele . Det gaar foran den Stcerke i Kampen , og det bliver tilbage hos de Faldne for at gyde Balfom i deres blodende Vunder . Det vejleder gjennem Tiden , og det aabner Evighedens Dsre . For dette sit Ord har Gud nu vistnok beredt Menneskesjæle og Menneskehjerter som sine sande Templer . Hvor der er et Menneskehjerte , der banker i Tro , en Menneskesjel , der bsier sig i Ydmyghed , en Menneske aand , der lofter sig i Tilbedelse , der er ogsaa et Guds Hus , hvadenten de omtumles for Vindene paa det gyngende Skibsdek , eller de grave sig Gange gjennem lordens

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014111808028'target=_>Dietrichson, L., 1877, Paa studierejser</a><h4>648</h4>lagttagelse , at Maria med Barnet allerede var afbildet mange Gange i det andet Aarhundrede , Dette er unægtelig saa meget mere paafaldende , sora Katakomberne ellers blot idetmindste paa denne Tid <b>vise</b> os Billederne af de i Rom afdøde Helgener ; men naar man med Katolikerne gaar saa langt , at man herved vil bevise Madonnadyrkelsens Tilværelse hos det andet Aarhundredes Kristne , saa skyder man langt ud over <b>Maalet</b> . Thi hvad der bevises af Billederne , er blot , at Herrens <b>Moder</b> omfattedes af hans Disciple lige fra Kirkens ældste Tider med samme ærbødige Kjærlighed som de Martyrer , der vare „ hans Brødre og Søstre “ ; meu ligesaa lidt som Helgendyrkelsen kan udledes af disse Gravbilleder , ligesaa lidt kan Madonnas Billeder vidne om andet end en naturlig sønlig Kjærlighed til Kristi <b>Moder</b> , den første , som havde lidt for Kristi Skyld ,

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006111400030'target=_>Dietrichson, L., 1877, Fra min Vandringstid</a><h4>648</h4>Maria forekommer snart med , snart uden Barnet . opdaFeiB6n ak St . Priscillas Katakombe medførte den interessante lagttagelse , at Maria med Barnet allerede var afbildet mange Gange i det andet Aarhundrede . Dette er unZ ? AteliA saa meget mere paafaldende , som Katakomberne ellers blot — idetmindste paa denne Tid — <b>vise</b> os Billederne af de i Rom afdøde Helgener ; men naar man med Katolikerne gaar saa langt , at man herved vil bevise Nadonnadyrkelsens Tilværelse hos det andet Aarhundredes Kristne , saa skyder man langt ud over <b>Maalet</b> . Thi kvad der bevises af Billederne , er blot , at Herrens <b>Moder</b> omfattedes af hans Disciple lige fra Kirkens ældste Tider med samme ærbødige Kjærlighed som de Martyrer , der vare n hans Brødre og Søstre " ; meu ligesaa lidt som Helgendyrkelsen kan udledes af disse . Gravbilleder , ligesaa lidt kan Madonnøs Billeder vidne om andet end en naturlig sønlig Kjærlighed til Kristi <b>Moder</b> , den første , som kavde lidt for Kristi Skyld .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009081903024'target=_>Hartwig, Georg, 1860, Naturen og Menneskelivet i det høie Norden</a><h4>589</h4>Efterat vi en Tidlang havde underholdt os med hinandcn paa denne Maade , bleve vi vcnligst indbudne til at overvcere en Buestydning . Ude paa Markm staae de unge Folk allerede opstillede i Rcekker , og prpve deres Buestrcenges Styrke . Bag ved Inglingene staner en Rcekke blomstrende <b>Piger</b> . Her stal det <b>vise</b> sig , om han besidder den Legemkraft , der er saa uundvcerlig for den Vilde ; thi det kommer ikke saa meget an paa at trceffe <b>Maalet</b> , som paa at <b>vise</b> sin Armstyrke . Derfor blive Pilene som oftest studie ud i Luften , hvor den ene jages af den anden , som Spurven af Falken . Med henrykte Blik forfplge <b>Pigerne</b> hver godt afstudt Piil og hilse den lykkelige Stytte med et vedholdende kee ! Hvor behagcligt dette lyder i Inglingens Aren , derom vidne hans rpdmcnde Kinder tilstrcekkeligt."

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092303046'target=_>Platou, Ludv. Stoud, 1830, Ludv. Stoud Platou's Lærebog i Historien</a><h4>1059</h4>ges ved Gift efter Livias Foranstaltning . lulia blev derefter formalet med den alderstegne Kriger Agrip ) pa , og <b>Moder</b> til zdeSonner og 2 Dsttre . De 2 de « loste Sonner adopteredes , afAugust , mendenenedsde i Gallia og den anden i Asia , begge ved underkjsbte Lceger . Livia og hendes Son af fsrstelEgtestab , Ti « berius , ncerme sig derved <b>Maalet</b> for deres Bestroebelser . Agrippa doer , og lulia tvinges til at cegte Tiberius , men bliver ved Forfsrelser udsvcevende . Den af Li » ria udtoenkte List virker saalcenge paa den gamle Au » gustus , at han , ttobs sin Faderkjcerlighed , forviser lulia til den ode De Pandataria ved Campania . Saaledes bersuet fast alle Slcegtninge maa August adoptere Stedssnnen Tibcrius , en modig men ond Mand . Efter 44 Aars Regjering oser Augustus i d 19 de Nola i Eampania , hvorpaa Livia iler med Tiberius til Norna for at tåge den rige Arv i Besiddelse . Un . ' e . Chr , der Augustus ' s rolige og lykkelige Statssiyrelse for , ' svandt aldeles den republikanske <b>Aand</b> . En nye Slcegt opstod , der fra Barndommen af var vant til , og fandt sig vel ved Eneherskerens Regjering . Det kraftlose , indtil usselt Krpberie lydige Senat vorder Despoternes Redstab , ligesom Prcetorianerne deres Ststte .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008091800028'target=_>Jacobsen, Jacob, 1859, Den bibelske Historie i Udtog efter de hellige Skrifter</a><h4>378</h4>Det stal hverken vcere mit eller dit , del det . Da sagde Kongen : giver Barnet til den Forste ; thi hun er dets <b>Moder</b> . Og denne Dom Holtes over al Israel , og de frygtede for Kongen ; thi de saa , at Guds <b>Visdom</b> var i ham til at holde Dom . " I Salomos Tid herstede der bestandig Fred i Landet , saa at hver Mand i luda og Israel kunde bo tryggelig under sit Vintra ? og Flgentrcc . Og da Landet saaledes var velsignet med Fred , da tcenkte Salomo paa at bygge Herren et Hus , efter Herrens Ord til hans Fader David . Og Salomo byggede det store og prcegtige Tempel paa Moria Bjerg i Jerusalem . Det blev fuldfort i 7 Aar og indrettet efter Tabernaklets <b>Vis</b> , indvendig beklcedt med Cedertrce og rigeligt prydet med Guld . Det var 60 Alen langt , 20 Alen bredt og 30 Alen hoit og sammensat af hele Stene . Da Bygningen var fcerdig , indviede han den med Bon til Gud og sagde : „ Herre / Israels Gud , der er ingen Gud som du i Himmelen og paa Jorden . Se , Himlene og Himlenes Himle kunne ikke rumme dig . langt mindre dette Hus , som jeg har bygget . Men vend dit Ansigt til din Tjeners Bon , at dine Dine maa hvile over dette Hus Dag og Nat . " Fremdeles bad han , at Herren vilde hore den Retfærdiges Bon , men betale den Ugudelige efter hans onde Veie , og at han vilde hore Folkets Bsnner , nåar de omvendte sig til ham og bekjendte sine Synder , eller nåar de i Misvækstens Aar og under andre Landeplager paakaldte ham i dette Hus . Ja han bad ogsaa for den Fremmede , som Me nar as Israels Slcegt , at Gud ogsaa vilde hore hans Suk , nåar han gik ind i dette Tempel for at bede , paa det at alle Folk paa Jorden maatte kjende Guds Navn og frygte ham , ligesom Israel frygtede ham . ( 1 Kong . 8 Kap ) . Da Templet var fcerdigt , offrede Salomo et stort Takoffer, og Kongen og Folket holdt Hoitid for Herren il 4 Dage . Selv en Dronning fra Saba i Arabien kom for at fatte Salomos beromte <b>Visdom</b> paa Prove ; men der var ilke et Ord skjult for Kongen , som han ei kundgjorde hende . Og da hun erfarede hans <b>Visdom</b> og saa Huset , som han havde bygget , og al hans Herlighed , da sagde hun : „ Salige ere dine Mcend , salige dine Tjenere , som stedse staa for dit Ansigt . " ( Mach 12 , 42 ) . Men uagtet den store <b>Visdom</b> og Gudsfrygt , hvormed Salomo begyndte sin Regjering , faldt han dog fra Herren i sine sidste Dage , idet han tog

<h4>527</h4>2 Tim . 3 , 5. Flyer dem , som have Gudfrygtigheds Skin , men forncrgte dens Kraft . Medens Folket i Almindelighed holdt sig til Jesus , sogte <b>Visdom</b> og Trost af hans guddommelige Ord , og Hjalp og Lcegedom af hans barmhjertige Haand , var det fornemmelig de <b>Vise</b> og Mcegtige , der hadede og forfulgte ham . ( Mach . 11 , 25 , Cv . paa 26 de Send . eft . Trin . 1 Kor . 1 , 26. 27 ) . Blandt disse Jesu Fiender vare iscer de saakaldte Farisæer , hvilket Navn betyder de Udmcrrkede , idet de adskilte sig fra Andre ved en <b>vis</b> udvortes Strenghed i Opfyldelsen ikke alene af Mose Lovs Bud , men ogsaa af selvopfundne Forstrifter , medens de i Hjertet vare onde og foragtede Andre . Derfor sagde Kristus om dem : „ Paa Mose Stol sidde Skriftkloge og Farisccer ; de binde svare Byrder , som man vanskelig kan bare , og lagge dem paa Menneskenes Skuldre , men selv rore de dem ikke med en Finger . De gjore alle sine Gjerninger for at sees af Menneskene , ' og ville gjerne have de ypperste Sceder i alle Selskaber og Forsamlinger. Men ve Eder , siger han , I Oienstalke , I lukke Himmeriges Rige for Menneskene . Selv gaa I ikke derind og hindre dem , som ville gaa derind . I bede lange Venner for et Syns Skyld , men hemmelig opcede I Enkers Huse . Ve Eder , thi I ere som kalkede Grave , der synes deiliae udentil , men som indentil ere fulde af dode Ben og Urenhed . Saaledes synes og I udvortes retfærdige for Menneskene men indvortes ere I fulde af Hykleri og Uret . ( Matbl 23 de Kap . ) . Et andet Parti af Jesu Fiender faldtes Saduscrerne , d . e . de Retfcrrdige . Disse troede ikke paa nogen Opstandelse efter Doden , ikke heller at der gaves nogen <b>Aand</b> eller Engel . ( Ap . Gj . 23 , 8 ) . Deres udvortes Mrbarhed var saaledes kuns en vantro Egenretfcrrdighed . Disse Sadusceer opkastede engang saadant Spsrgsmaal til Jesus : „ Naar en <b>Kone</b> har havt 7 Brodre tilcegte efter hverandre , hvem af disse stal hun da tilhore efter Doden ? " Jesus svarede : „ I fare vild , idet I hverken kjende Skrifterne ei heller Guds Kraft ; thi i Opstandelsen stal der hverken tages tilcegte eller gives tilcegte , men de stulle vcere lig

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009020600075'target=_>Nissen, R. Tønder, 1864, Verdenshistoriens vigtigste Begivenheder i fragmentarisk Fremstilling</a><h4>789</h4>Da Spåmens Magt dalede , hccvede sig i dets Sted Frankrige . Dette styldtes for en vcesentlig Del den franske . Statsmand Richelieu . Armand Johan du Plessis , Herre til Richelieu , var yngste Son af en fransk Adelsmand. Han var egentlig bestemt for den militcere Stand ; men da han havde Udsigt til at kunne gjore det bedre ved at troede i Kirkens Tjeneste , begyndte han at stuoere Theologi og blev , 22 Aar gammel , viet til Biskop . Men hans Strceben stod til et endnu hoiere <b>Maal</b> . I Frankrige regjerede dengang Ludvig 13 , Ssn af Henrik 4. Ludvig var svag og befattede sig ikke synderlig med Regjeringen ; hans <b>Moder</b> , Maria af Medici , havde styret Riget , indtil han blev myndig , og hun og hendes Indlinger besad ogsaa sioen megen Indflydelse . Richclieu vandt Enkedronningens Indest og Tillid . Hun fik ham ind i Statsraadet , uagtet de celdre Medlemmer sogte at hindre det , da de frygtede hans overlegne <b>Aand</b> . Et halvt Aar senere var han Raadets fsrste Medlem og Frankriges egentlige Regent . Paven gjorde ham til Kardinal . — Under hele sin Statsstyrelse havde " han for 2 > ie at udvide den kongelige Myndighed og at ydmyge det spansk - ssterrigske Hus . Da han kom til Roret , opfsrte de Store sig , som om de ikke vare Undersaatter , og Befalingsmcendene i Provindserne , som om de vare uafhcrngige Herrer . Men Nichelieu kvede de Store og befceitede Kongens Myndighed . I dette Diemed bekrigede han ogsaa Hugenotterne ; thi de havde sceregne Rettiqheder og kunde derfor , naar de vilde , trodse den kongelige Vilje . Han fratog dem deres Sikkerhedspladse , men indrsmmede dem fuldkommen Religionsfrihed . For at ydmyge Spanien og Osterrige blandede han sig i alle Europas politiske Spsrgsmaal . Til den Ende underststtede

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010062903002'target=_>Steffens, Henrich, 1840, Hvad jeg oplevede</a><h4>400</h4>Gang satte ham i Forlegenhet » , og bragde ham i en selsom , ja betcenkelig Stilling . Til de <b>Piger</b> , der iscrr tiltrak sig min Opmcerksomhed , og hvis Ncrrvcerelse var mig pderst behagelig , horde Dsttrenc af en borgerlig Minister . De bleve mig aldeles fjerne ; jeg erindrer ikke , at jeg har taelt eet Ord med dem . Men jeg var lykkelig i deres Ncrrvcrrelfe , og kaldte gjcerne det behagelige Billede tilbage i min Erindring . Den Ene cegtede en Prinds . Ligesaa fjernt stode mig Dsttrene af en theologisk Professor , der baaoe ved deres Skjsnhed og deres <b>Dannelse</b> gjorde Opfigt . De bleve bortgivtede til Venner , jeg saae dem mange Aar senere som Hustruer , som meget elfivcerdige Msdre, og de havde lcenge beholdt deres Ungdoms Inde . Saaledes horde ogsaa blandt de <b>Qvinder</b> , som jeg erindrer mig med reen Glcede , en Vens Brud , en Blondine , der vel allerede kunde verre 24 til 25 Aar gammel . Hun kom ofte i Professor Bangs Huus , og blev baade af hans Sonner og af mig behandlet med den stsrste Agtelse . Hun udmcrrkede fig ikke alene ved sin <b>Dannelse</b> , men ogsaa ved sin <b>Aand</b> . Ungdommeligheden var forenet med en stille Anstand , jeg kunde ncesten sige Bcerdighed . <b>Pigerne</b> vedligeholdt i mig Billedet af reen <b>Qvindelighed</b> ; jeg Huao jeg opleveoe » 11. 9

<h4>405</h4>Bestandigt har en Fremstilling af rsrende Optrin , selv naar jeg maa dadle Digteren og ingenlunde er tilfreds med Skuespillerne , en stor Magt over mig , ja den kan undertiden rsre mig til Taarer ; men denne Rystelse var dybere ; det var ganste afglort min forste Kjcerlighed . Jeg saae hende anden Gang som Kathinka i Pigen fra Marienburg . Dette Stykke , der dog egentlig er af et ringere Slags , fsrer os paa en ret unaturlig Maade en Underofficiers Datter ligeoverfor en Keiser , som et Msnster paa Skjsnhed , Dyd , Klogsiab , Heroisme , ja Viisdom og paa en <b>vis</b> Maade ogsaa <b>Lcerdom</b> . Jeg vilde have leet ad dette moralste doctrinaire Prceparat i en ung <b>Piges</b> Skikkelse , men nu syntes alt mig naturligt . Kun da hun til Slutningen , overflydende af <b>Lcrrdom</b> citerede en af Karthagernes Love , og kun med Moie kunde udtale det fremmede Ord , maatte jeg rigtigtnok lee , men selv denne <b>qvindelige</b> Übehcrndighed forsgede hcndes Elstvcrrdighed .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008052302002'target=_>Bugge, Søren Bruun, 1837, Lærebog i de romerske Oldsager</a><h4>939</h4>sig selv , at den i Nationens celdsie « dannede Tider var mere legemlig , end <b>aandelig</b> , og bestod fornemmelig i at vcrnne de Unge til en god Statsborgers Sieder og Handl lemaade . Bsrnene bleve opammede og opfostrede af deres <b>Modre</b> . Efterat Barndommens forsie Aar vare tilbagelagte , toge de Deel i Foreldrenes Forretninger ; Drengen ledsagede sin Fader paa Forum , i Comitierne , i Senatet , , paa Ågeren , stundom ogsaa i Felten , og Pigen svedes af Moderen ien <b>Huusmoders</b> Sysier . Paa vii denstabelig <b>Dannelse</b> blev ikke videre tcrnkt , end forsaai vidt Bornene , for at lcrre Sproget og de allerforsie Vegyndelsesgrunde , bleve hensendte i Skoler , der heed tadernZL litterariN 8. lu6i litterarii , og hvor ds for en ringe Betaling bleve underviste af Lcerere , som heed lullim » KiBtri Ved denne <b>Dannelse</b> lod man det da forstsrstedelen forblive , naar undtages den Undervisa ning , som Prcrsierne fik i Neligionslcrren , men som ikke gaves i Nom , men i Etrurien ( § 89 ) . Men eftersom Smagen for Videnffaber i den fsli gende Tidsalder tiltog , begyndte man ogsaa at lcegge mere Vlrgt paa Bornenes litteraire <b>Dannelse</b> , i hvilken Hem

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009052803025'target=_>, 1853, Portræter af mærkelige Nordmænd</a><h4>1990</h4>ham i Mestcrletticn . „ Intet Trin siden ( har Brun selv fortalt ) virkcde med saadan V ^ lde paa min <b>Aand</b> . Sclvfolelse og Kappelyst vaagncde med cet . Jeg dromte for kun om Ski og lagl , om at sanke Beer og at fiske Forcllcr , og satte ingen Priis paa den Smule Latin og Grcesk , jeg havde lcert og lcere skulde . Jeg var flittig i 2 Aar kun for at behage Forceldre og en Broder , jeg elskede . Jeg blev det siden for at glimre blandtLigcmcend ; jeg glimrcde ikke lcenge , forend jeg begyndte at elske Homer , Virgil og Horats , og desuden ethvert Fremsiridt til min <b>Aandsdannclse</b> . " Efter to Aars utrccttclige Flid var Brun moden til Universitetet , men da Forceldrene syntes , at han endnu var for ung til at besoge den fristende Hovedstad , saa blev han det tredie Aar i Mestcrlcctien , Iccste Genesis , hele Iliaden og Hesiod . Ofte plciedc han at sige :  Icg har aldrig vceret lcrro i mine egne Tanker uden til Examen Artium . " Ved i Skolegangen at have Kostdage hos gode Borgere af hans Familie , saae han i Huset hos Grosserer Otto Beyer en smuk <b>Pige</b> , ved Navn Ingeborg Lind , en Broderdattcr af Beycrs forstc <b>Kone</b> . Endskjont hun cndnu blot var 16 Aar og han selv 17 Aar gammel , forlovede han sig dog med hende . Han beilede i Vers .  De vare kun daarlige ( sagde han ofte ) , men de lonnede sig bedre , end de mere udarbcioede siden . " I Aaret 1763 reiste Brun til Kjobcuhavns Universitet og tog Examen Artium , Aaret cfter anden Eramcn og lcerte Fransk . Endnu i samme Aar vendte han tilbage til Throndhjem , hvor han blev Huuslcerer hos Kammcrraad Mcinckc . Nu ovede han sig ogsaa iat prcedikc , hvortil han solte et scerdcles Kald . I 1767 fulgte han Meinckes Son til Soro , og vilde ved samme Leilighed tåge Attestats , hvorfor han lcestc i 3 Maancdcr Nat og Dag uden Manuduktcur , og fik Attestats med den Karakteer , han selv havde attraaet , nemlig I ^ un ocilU « mn6ncluB ; thi han havde ikke lcest til Karaktccr , som det da kaldtes ; Candidaten kunde nemlig dengang selv anmelde den Karakteer , han troede at have gjort sig vcerdig til . Brun reiste derpaa strar tilbage igjen til Throndhjem , hvor han tilbragte 3 Aar med at informere og prcedike . I 1770 frcmtraadte han offentlig for forstc Gang som Digter ; hans forste Arbeide var poetisk Prosa ( indfort i Samlingen af hans mindre Digtc S . 103 ) , og strar efter strev han , for at lykonske Biskop Guu ner us til hans Hjemkomst fra Nordlandene, Digtet : Naturens Navnedag . Begge disse Prover blcve med Roes omtalte ide offentlige Tidender . I 1771 blev Biskop Gunncrus pludselig beordret af Struensee til Kjobenhavn , for at reformere Universitetet , og Brun blev af Gunncrus antaget som Amanuensis og fulgte med ham til Kjobcnhavn . Da imidlertid Gunnerus ei saa godt kunde bruge Brun til de Forretninger ,


<h2>“Kvinnens væren” [Woman’s being]</h2><h3Kvinde_, Væren_, Arbeid/Kall_, Egenskaper_</h3><h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009091403021'target=_>Luther, Martin, 1864, Dr. Martin Luthers Udførlige Udlæggelse af første Mosebog</a><h4>4456</h4>og Uro , end et forferdet Hjerte , forn blegner blot det ser et Lyn , ja blot det horer et Blad rasle . 297. Saaledes have ogsaa Jakobs Sonner en ond <b>Samvittighed</b> , derfor forferdedes de saa gruelig og frygte for Guds Hevn . De ere bange for , at han vil staa til med Lyn og Torden , og dog beljende de endnu itte Synden . Men det er en underlig Ting : Samvittigheden foler Guds Dom og Vrede , og dog stammer den sig for at gjore Bod og beljende Synden , om den saa end flttlde blive stikkende i evig Fordommelse og Pine . Og dette ster paa Grund af Arvesynden , som man ogsaa kan se paa Adam . Thi da han allerede havde ladet sig overvinde og flyede for Gud , fordi han vidste sig skyldig i Faldet , var dog Arvesynden i ham endnu stum og kunde itte taale , at dens Oren aabnedes , hvorvel Guds Nost havde lydt for ham : Adam , hvor er du ? 1 Mof . 3 , 9. 298. Erasmus skriver paa et Sted om en spartansk Pige , som var bleven besvangret ved Horeri ; da hun havde fodt sit Barn , vilde hun heller drebe det end beljende sin Synd . Denne hendes <b>Gjerning</b> udlegger han som en stor Kraft og Mandighed hos hende , uagtet det dog meget mere var en Svaghed , at hun heller vilde styrte sig i Fare for Dod og evig Fordommelse end beljende sin Synd . Saa stor er Arvesyndens Galstab , Raseri og Afsindighed , som itte vil lade sig beskjemme eller tillade , at dens Skam sees . Det er ganske imod Syndens <b>Natur</b> og Vesen at beisende , at man har syndet . 299. Saaledes forteller man om St . Martinus ^ ) , at han , da Djevelen spottede og dadlede ham , fordi han var saa snar til at forlade de verste og syndefuldeste Mennester deres Synder , stal have svaret : kunde du kun sige nf Hjertet : det gjor mig Ondt , at jeg har syndet , saa vilde jeg absolvere endogscta dig og erklere dig fri for Synder . Thi ogfaa Djevelen vilde blive salig ligesacrvel som nogen Anden , hvis han af et rent Hjerte kmtde sige : ak Gud ! forbarm dig over mig og ver mig naadig ! 219. Absolutiouen og Syndenes Forladelse er sitter , kun at du af Hjertet erkjender og bekjender , at du har syndet . Men det kan ikke Djevelen . Adam og Eva kunde det heller ikle strax . Thi saa sagde Adam i Mos . 3 , 10. : jeg horte din Rost i Haven , og jeg frygtede ; thi jeg var nogen . Han siger itte : fordi jeg havde overtraadt dit Bud , forent , Gnd felv driver ham derhen og fremdeles sporger ham V . 11. : hvo gav dig tilkjende , at du var nogen ? har du edet af det Tre , om hvittet jeg bod dig , at du ftnloe Mc ede deraf ? Men endnu vil han itte vere stinkende ; endnu erkjender og bekjender han itte den Synd , som foreholdes ham , men giver <b>Kvinden</b> Skylden V . 12. : <b>Kvinden</b> , forn du gav at vere med mig , hun gav mig af Treet , og jeg aad . Se , stjont hatt allerede var forskrekket og bange og stjalv , fordi han folle , at han havde syndet , bekjender han dog endnu ikke Synden . 211. Et lignende Exempel herpaa have vi ogsaa i Savl , der siger 1 Sam . 15 , 30. : jeg har syndet , men , Kjere , er mig nu for mit Folks og for Israel ; forn om han vilde sige : hjelp mig at tildette Synden . Saadant duer slet itte for Mennestene engang . Thi at begcta en Synd og saa fornegte den , gjor Synden dobbelt , ja syvfold

<h4>4530</h4>269. Benjamin alene tenker han nu paa ; for ham berer han stor Omsorg ; han foretrekker ham fremfor alle de Andre , og dertil har han ogsaa vigtige og nodvendige Grunde , idet han nemlig var Rachels Son , og Forstefodseletts 3 - Ere tilkom ham . Men om alle de Andre kunde han intet Godt tenke . Jeg fer , siger han , at alle Eders Naadstllgninger og Foretagender ere uheldige og faa en daarlig derfor vil jeg Me saa let lade Beujamin blive udsctt for saadan Angst og Fare . Men dog vil jeg ikke hindre Eder Andre , eller Nogle af Eder , fra at drage ned . 279. Denne Jakobs Frygt og Mistanke var naturlig og mennestelig. Thi de , som befinde sig i Anfegtelfe , gjore sig altid de verste Tattker og Forestillinger , ligefom de , der tugtes og ere ulykkelige , heller itte vente ttoget Godt af Gud ; thi for deres Tattker fremstille sig kun saadanne Ting , hvorved deres Elendighed og Ulykke bliver storre ; ja de mene , at alle Omstendigheder og Tilfelder kun bringe dem Ulykke og den hoieste Fare . 271. Og det er Djevelens <b>Gjerning</b> , som han fornemlig pleier at ove i en stattels <b>Samvittighed</b> , forn er styldig for Gud , den cmgriber han paa en forferdelig Maade med giftige Bid og sine gloende Pile . Ligesom ogsaa paa den anden Side , nåar Alt staaer vel til med os og gaacr os efter Onste , Ingen tenker paa , hvor ner da Doden , Djevelen og allehaande Fare kan vere os , som hin gamle Mand i Komedien viselig siger : nåar det gaacr os vel , bor vi Alle allermest tenke paa , hvorledes vi stulle stikke os i Ulytteu . Hvo der efter en Neise kommer hjem , bor altid tenke sig , at enten hans Son har syndet , eller hans <b>Hustru</b> er dod eller hans Datter syg ; thi dette hender hver Dag og kan lettelig ramme Enhver , hvorfor hatt itte maa tro , at Saadant horer til Sjeldenhederne . Men Alt , som ster bedre end vi havde haabet , maa man anse for en Vinding . 272. Mett dette falder os itte ind , nåar ingen Ulykke rammer os ; ja , vi pleie da at snorke og vere sikre , som om det slet ingen Nod havde ; ligesom vi i Anfegtelsen , og nåar Straffen kommer , skjelve og beve . Thi saaledes bringer den menneskelige <b>Natur</b> det med sig . Derfor ere vi i begge Tilfelde usle , elendige Mennesker . Ofte paaoigte vi os selv af een Synd , for hvis Skyld vi straffes , utallige andre Synder . Og atter , nåar vi ere trygge og have Fred , saa selv om vi ogsaa da ere belessede med mange Synder , tenke vi dog end Me paa een , frygte heller Me for nogen Fare . 273. Denne Svaghed hos os stttlle vi erkjende , nemlig at vor <b>Samvittighed</b> , nåar dett er sitter og glad , samler sig utallig megeu Glede , men ttaar den er sorgmodig og bedrovet , kan den Me samle sig Andet end idel Skade , Jammer og Ulykke . Thi dette vise saavel Exemplerne i dett hellige Skrift forn Enhvers egen Erfaring . Naar mit Hjerte er befvceret med Sorgmodighed eller Vrede , synes mange Tanker at vere ganste sande ; nåar jeg derimod atter kommer til mig selv , kan jeg lettelig forståa Vildfarelfen . 274. Jakobs Sonner opholdes hjemme lengere end de vel havde ventet . Thi de tenkte , at de strax igjen stulde drage ned til Egypten , foråt holde sit Lofte og befri sin Broder . Og de havde gyldig Grund nok til snart igjen at komme til Egypten ; men deres Fader hindrer dem fra at fuldbringe sit Forehavende . Thi han vil heller udsette Simeon for Fare end Benjamin , og kommer frem med en god dialektisk

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2008082512001'target=_>Luther, Martin, 1863, Dr. Martin Luthers Udførlige Udlæggelse af første Mosebog</a><h4>685</h4>og Eva ofte med hinanden , og derved lindrede de al timelig Modgang og Ulykke og trostede sig . 193. Saaledes skulle ogsaa vi ofte forestille os det tilkommende Livs usigelige Skatte , og ved faadaune Tauker gjore al timelig Modgang og Kjodets Besvcer mild og let , som vi se , at Paulus gjor 2 Cor . 4 , 17. 18. : vor Trengsel , som er stakket og let , bringer os en evig og over al Mao.de vigtig Herlighed , idet vi itke have de synlige Ting for Die men de usynlige ; thi de synlige ere timelige , men de usynlige evige . 194. Sig mig uu , om ikte de let og taalmodig skulde bcere al timelig Ulykke , saaledes som den kau forefalde , og sige , at dette er en Faders og ikke eu Dommers eller Tyraus Vrede , de som saaledes gribe det evige Livs Haab og tro Guds Tilsagn , som derncest ogsaa se hen til den timelige Velsignelse , at Gud har iudrommet os hele denne Verden , foråt vi skulle bruge den , har givet os <b>Hustru</b> , Born , Hus og Hjem , opholder ogsaa og foroger alt dette med rig Velsignelse . Derimod kan man ved Slangen se en strceng Dommers Vrede ; thi den forjcettes ikke alene ingen Hjcelp eller Forlosning , meu den trues ogfaa med , at dens Hoved stal kuuses . Denne Vrede har Djceveleu nok faaet fole og foler den endnu ; derfor larmer og raser han for den yderste Dag med saadan Harme og Vold imod Kirken og Guds Son . 195. Saa er dette uu vel en Trudsel , hvorved Gud truer med at straffe Eva ; men midt i Straffen fremlyfer en usigelig Barmhjertighed , som saaledes avreiser og troster Eva , at hun ogsaa i sin Ulykke er ved godt <b>Mod</b> og glad . Nu fole vi ogfaa selv , hvor fornoden Straffen er os til at tcemme Kjodet . Thi hvorledes kunde vi ydmyges , hvis ikke denne <b>Natur</b> blev trykket og tugtet faaledes med Straffens Byrde ? Derfor maa Eva og enhver Kviude i sit <b>Kald</b> og sin <b>Gjerning</b> visselig bcere sit Kors og erfare megen Jammer , Angst og Nod , og med Smerte undfange og fode . 196. Men her staaer det Ord Rav , hvilket betyder Storhed og Mlengde , saaledes at man stal forståa det ikke alene om stor , men ogsaa om megen og mangehaande Jammer og Smerte , hvilken Eva aldrig havde faaet fole , hvis hun ikke var falden i Synden . Og Gud truer ifcer Uudfangelfen og Fodfelen . Men ved Undfangelsen forstaaes hele den Tid , hvori det uudfangede Foster bceres i Moderlivet , og mangehaande svcere Tilfcelde og Sygdomme indtrceffe . Thi strår foler man Hovedvine , Svindel og en forunderlig Vcemmelse for Mad og Drikke , bestandig og besvcerlig Ovkastelse , Taudvine og Uorden i Maven , hvoraf den Lyst kommer , som man kalder Barselkvinders Lyft , idet man faaer Lyft paa saadanne Ting , som man har Afsty for , nåar man er frist . Men nåar nu Fosteret er dannet i Moderlivet , og Fodfelen er forhaanden, da folger den yukeligste Jammer , idet Moderen maa sode Barnet med den storste Fare , ja ncesten med Dodens Angst og Nod . 197. Naar nu Hedmngerne , som Intet forståa eller vide om Gud og hans Gjerninger , fe dette , saa tåge de en saadan Forargelse deraf , at de stutte , at en vis Mand ikke kan tåge sin nogen <b>Hustru</b> vaa Grund as denne Jammer og Elendighed . Thi det <b>kvindelige</b> Kjon bliver meget ydmyget og vlaget , og maa bcere langt haardere og tungere Straffe end Manden . Thi hvad har en Mand at lide vaa sit Legeme i Sammenligning med en <b>Kvinde</b> ? Men efterdi ved 2 Egtestanden Manden vaa en Maade faaer sin Del af disfe <b>Kvindens</b> Straffe ( thi en from Mand kan

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014082608023'target=_>James, John Angell, 1872, unge Kvinde</a><h4>429</h4>ganske forskjellig <b>Natur</b> . Jeg ser ikke her for mig den <b>Kvinde</b> , som ledes bort fra Forældrehjemmet af den Haand , som senere skal blive hendes Støtte og Beskyttelses eller den , som forlader det , støttet ved en Arm , som vedvarende vil voere rede til at rcekkes hende ; men den , som forsvarsløs og sorgmodig trceder ud i Verden for ensom at gaa Livets Farer , MKe og Bekymringer imM . Det er sørgeligt nok at se en ung Mand forlade sin Faders Hns for at „ cede sit i sit Ansigts Sved ; " hvor meget mere da at se en <b>Kvinde</b> saaledes vandre bort for nt sFge sit Ophold ! Hvad er hun andet end et ungt Lam , som vover sig ud i en Men , opfyldt af glubende Ulve , eller en spced Due , som forlader sit Rede for at flyve bort blandt Gribbe og Arne ? Hvor mange ere ei de , som under Livets LB og midt iblandt dets OmverUnger , hvilke stundom ere saa sMgelige , gjerne se tilbage til Skilsmissens Stuud og udraabe : „ O <b>Moder</b> , hvor sørgelige, hvor snude vare ei de Anelser , som afpressede mig de Tllarer , jeg udgjß , da jeg maatte rive mig ud af dine trofaste Arme ! Mit Hjerte ' blMe da , og min Glades Sol gik ned for aldrig mere at oprinde . Ethvert Trin paa min Vane har vceret bestrøet med Torne , og enhver Forandring blot forøget min Bekymring . " I adskillige Tilfcelde er Skilsmissen fra Hjemmet nsdvendig fom FMge af forandrede Omstændigheder , fom ikke mere tillade DKrene at forblive der . Og de , som da have faaet en fortjcelet Opdragelse , omgivne af Pragt og Overflod , ere nu tvungne til at forlade det Hjem , hvor og <b>Arbeide</b> havde vceret fremmed for dem , for selv at fortjene sit daglige Vrpd . Man kan . ei andet end med Beundring se paa den unge <b>Kvinde</b> , som ved en saadan Leilighed , heller end at blive en Byrde for sine Forceldre og tilbringe sit Liv i gavnlM Sorg og Klage , staalscetter sig for <b>modig</b> at mMe de Forsagelser , som ere nødvendige i hendes forandrede Stilling , og som nedstigende fra sin forrige HMe , med sand Sjcelsstorhed bereder sig til ensom at betrcedc Livets stormende Sti . Ingen tcerende Fplelse af Fornedrelse , ingen falsk Undseelse over

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112101060'target=_>, 1866, Konkordiebogen, eller Den evangelisk-lutherske Kirkes Bekjendelsesskrifter</a><h4>1739</h4>Bud :  Hedre din Fader og din <b>Moder</b> " alene angaar legemlig ! Kjsd og Blod , hvorved man dog ikke ser hen til Kjsd og Blod , men til Guds Bud , hvori det er indbefattet , og for hvis Skyld Kjod og Blod kaldes Fader og <b>Moder</b> , ogfaa her ; om vi end ikke havde Mere end disse Ord :  Gaar hen og dsber " o . s . v. , saa maatte vi dog antage og gjere det som Guds Anordning . Men nu er ikke alene Budet og Befalingen der , men ogsaa Forjettelsen ; derfor er det endnu meget herligere , end hvad Gud ellers har paabudt og forordnet , med eet Ord : saa fuldt af Trsst og Naade , at Himmel og Jord ikke kan fatte det . Men der horer Kunst til at tro Saadant ; thi det mangler ikke paa Skatten , men det , hvorpaa det mangler , er , at man griber og holder den fast . 15. Derfor har enhver Christen hele sin Livstid nok at lcere og sve ved Daaben ; thi han har stedsc at arbejde paa , at han fast maa tro , hvad den tilsiger og bringer , Sejer over Djcevelen og Dsden , Syndernes Forladelse, Guds Naade , den hele Christum og den Hellig-Aand med Hans Ga ^ ver . Kort sagt , det er saa overvettes , at dersom den sirsbelige <b>Natur</b> kunde betenke det , da skulde den vel tvivle paa , om det kunde vere sandt . Thi tenk engang : oersom der etsteds var en Lege , som forstod den Kunst , at Folk ikke stulde ds eller , om de end dsde , dog snart igjen bleve levende og siden levedc evindelig , hvor vilde da ilke Verden lade Penge sne og regne ned over ham , saaat Ingen kunde komme til for de Rige ! Men nu faar her i Daaben Enhver uden Betaling en saadan Skat og et saadant Legemiddel bragt for sin Dsr , som opsluger Dsden og holder alle Mennesker ilive . 16. Saaledes maa vi betragte Daaben og fsre os den til Nytte , forat vi derved kunne styrke og trsste os , naar Synden og vor <b>Samvittighed</b> besverer os , og sige :  leg er dog dsbt ; men er jeg dsbt , saa er det mig tilsagt , at jeg stal blive salig og have det evige Liv baade til Sjcel og Legeme." Thi derfor ste begge disse Ting i Daaben , baade at Legemet overeses, hvilket ikke kan modtage Mere end Våndet , og at Ordet dertil udtales , forat Sjelen ogsaa stal kunne modtage Noget . Efterdi nu Vand og Ord tilsammen ere en Daab , saa maa ogsaa baade Legeme og Sjcel blive salige og leve evindelig , Sjelen formedelst Ordet , paa hvilket den tror , men Legemet, fordi det er forenet med Sjelen og ogsaa griber Daaben saaledes , som det kan gribe den . Derfor have vi hverken til Sjcel eller Legeme noget stsrre Klenodie ; thi derved blive vi ganste hellige og salige , hvilket ellers intet Levnet , ingen <b>Gjerning</b> paa Jorden kan opnaa . — Dette vere nu nok talt om Daabens Vesen , Nytte og Brug , saa Meget som her er tjenligt .

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2012052204016'target=_>Ahlfeld, Friedrich, 1869, Menneskelivet i Guds Ords Lys</a><h4>329</h4>— i de fyrste Barneaar ; thi Egenvillien , Ggensindigheden er allerede tidlig tilstede . Den er en Deel af Arvefvnden . Den er det fyrste Vidnesbyrd om Hovmodet i Menneskets Indre . Daarstab er bnnden til den Unges Hjerte , siger Salomo . Betragt dine Borns Liv . Neppe har deres Kraft udviklet sig ' lidt , forend de allerede kun altfor ofte ville Andet end Far og <b>Mor</b> ville . Endnn ftrend de kunne udtrykke sin Egenvillie med Ord , lcegge de den for Dagen med Fagter , Skrig og Uflikkelighed . Den findes hos Alle . Den vorer Dag for Dag . Vi kjende Moore , som allerede blive underkuede af sine smaae Born . De have ikke <b>Mod</b> og Kraft til strax i de forste Aar at kjcempe imod dem . Hvorledes stal det da blive i de modnere Aar , nåar Ulndigheden er bleven stor og stcerk , saaledes som hos Elis Sonner? Angrib den derfor tidligt . Medens Trceet endnn er et tyndt Sludd , kan det boies lige ; nåar det er blevet stort og stcerkt , er der enten stet Intet mere at foretage med det , eller man kncekker det . Enßcek kan man afdcemme og lede ind i en anden Bcek . Men forsoger man det Samme med en Elv , arbeider man enten forgjeves eller udfcetter sig for at overstylles eller bortriv es afdens Vande . Den hellige Skrift , der efter Guds naadige Raadslutning stal meddele Lcerdomme og Regler for ethvert Forhold i det menneskelige Liv , lcegger os i vor Tert ret alvorligt paa Hjerte at tåge tidligt fat paa dette <b>Arbeide</b> . Den gamle Sirak , der veeo saa god Bested om Huusvcesenet , siger i sin Bog : „ Har du Born , saa tugt dem , og boi deres Hals fra Ungdommen af . " Apostelen Paulus skriver til Forceldrene : „ IFcedre , opirrer ikle Eders Born ; men opfoder dem i Tugt og Herrens Formaning ! " Vor Herre og Frelser har vel ikte efterladt os noget Udfagn angaaende dette <b>Kald</b> , men i dets Sted en <b>Gjerning</b> , som taler hoiere end Ord . Omendstjont Guddommens Fvlde ogsaa boede legemligen i Ham , medens Han var Barn , boiede Han sig dog ydmngt under sine Forceldres Villie . Efter at Han i Jerusalems Tempel havde udtalt sig om sin himmelske Herkomst og svaret sin <b>Moder</b> : „ Bor jeg ikke vcere i min Faders ( Forretninger ) ? " gik Han ned med dem og kom til Nazareth og var dem underdanig . Her see I , Forceldre ; det maa saa vcere ; det er Guds Villie . Men dette see I ogsaa ved at kaste et Blik ind i Barnets <b>Natur</b> . Vel er det Guds Barn , men Guds uopdragne Barn . Endnu er det ikke ved nogen klar Tro bundel til Ham ; endnu er dets Hjerte ikke klart oplyst af den Hellig-Acmd ; endnu er dets Villie ikke helliget ved Guds Villie . Af Livets Begyndelse , af Udviklingens forste Aar afhcenger hele Fremtiden . Denne kan man ikke lcegge i

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014121208040'target=_>Ahlfeld, Friedrich, 1878, Menneskelivet i Guds Ords Lys</a><h4>318</h4>i de fsrste Varneaar ; thi Egenvillien , Egensindigheden er allerede tidligt tilstede . Den er en Deel af Arvesynden . Den er det fsrste Vidnesbyrd om Hovmodet i Menneskets Indre . Daarfiab er bunden til den Unges Hjerte , siger Salomo . Betragt dine Vsrns Liv . Neppe har deres Kraft udviklet sig lidt , forend de allerede kun altfor ofte ville Andet end Far og <b>Mor</b> ville . Endnu fsrend de kunne udtrykke sin Egenvillie med Ord , lcegge de den for Dagen med Fagter , Skrig og Uskikkelighed . Den findes hos Alle . Den voxer Dag for Dag . Vi kjende Msdre , som allereds blive underkuede af sine smaae Vsrn . De have ikke <b>Mod</b> og Kraft til strax i de fsrste Aar at kjcempe imod dem . Hvorledes stal det da blive i de modnere Aar , naar Ulydigheden er bleven stor og stcerk , saaledes som hos Elis Ssnner? Angrib den derfor tidligt . Medens Trceet endnu er et tyndt Skudd , kan det bsies lige : naar det er blevet stort og stcerkt , er der enten flet Intet mere at foretage med det , eller man kncekker det . En Back kan man afdcrmms og lede ind i en anden Back . Men forssger man det Samme med en Elv , arbeider man enten forgjceves eller udfcetter sig for at overskylles eller bortrives af dens Vande . Den hellige Skrift , der efter Guds naadige Raadflutning stal meddele Lcerdomme og Negler for ethvert Forhold i det menneskelige Liv , lcegger os i vor Text ret alvorligt paa Hjerte at tage tidligt fat paa dette <b>Arbeide</b> . Den gamle Sirak , der veed saa god Bested om Huusvcesenet , siger i sin Bog : „ Har du Bsrn , saa tugt dem , og boi deres Hals fra Ungdommen af . " Apostelen Paulus skriver til Fcrdrene : „ I Fcedre , opirrer ikke Eders Bsrn ; men opfeder dem i Tugt og Formaning ! " Vor Herre og Frelser har vel ikke efterladt os noget Udsagn angaaende dette <b>Kald</b> , men i dets Sted en <b>Gjerning</b> , som taler hsiere end Ord . Omendstjsnt Guddommens Fylde ogsaa boede legemligen i Ham , medens Han var Barn , bsiede Han sig dog ydmygt under sine Forceldres Vlllis . Efter at Han i Jerusalems Tempel havde udtalt stg om sin himmelske Herkomst og svarer sin <b>Moder</b> : „ Bsr jeg ikke vcere i min Faders ( Forretninger ) ? " gik Han ned med dem og kom til Nazareth og var dem underdanig . Her see I , Foraeldre ; det maa saa voers ; det er Guds V ' llie . Men dette see I ogsaa ved at kaste et Blik ind i Barnets <b>Natur</b> . Vel er det Guds Barn , men Guds uopdragne Barn . Endnu er det ikke ved nogen klar Tro bundet ? er dets Hjerte ikke klart oplyst af den endnu er dets Villie ikke helliget ved Guds Vlllie . Af Lwets Begyndelse , af Udviklingens fsrste Aar afhcenger hele Fremtiden . Denne kan man ikke Icegge i

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2006112101017'target=_>Ahlfeld, Friedrich, 1866, Menneskelivet i Guds Ords Lys</a><h4>258</h4>— i de fyrste Barneaar ; thi Egenvillien , Egensindigheden er allerede tidligt tilstede . Den er en Dccl af Arvesynden . Den er det fyrste Vidnesbyrd om Hovmodet i Menneskets Indre . Daarskab er bunden til den Unges Hjerte , siger Salomo . Betragt dine Bsrns Liv . Neppe har deres Kraft udviklet sig lidt , smend de allerede kun altfor ofte ville Andet end Far og <b>Mor</b> ville . Endnu fprend de kunne udtrykke sin Egenvillie med Ord , lcegge de den for Dagen med Fagter , Skrig og Uskikkelighed . Den ftndes hos Alle . Den vorer Dag for Dag . Vi kjende Mpdre , som allerede blive underkuede af sine smaae Bsrn . De have ikke <b>Mod</b> og Kraft til strax i de fsrste Aar at kjcempe imod dem . Hvorledes skal det da blive i de modnere Aar , naar Ulydigheden er bleven stor og stcerk , saaledes som hos Elis SMner? Angrib den derfor tidligt . Medens Trceet endnu er et tyndt Skudd , kan det bmes lige ; naar det er blevet stort og ftcerkt , er der enten slet Intet mere at foretage med det , eller man kncekker det . En Back kan man afdcemme og lede md i en anden Boek . Men forspger man det Samme med en Elv , arbeider man enten forgjeves eller udscetter sig for at overfkylles og bortrives af dens Vande . Den hellige Skrift , der efter Guds naadige Raadslutning skal meddele Lcerdomme og Regler for ethvert Forhold i det menneskelige Liv , lcegger os i vor Text ret alvorligt paa Hjerte at tåge tidligt fat paa dette <b>Arbeide</b> . Den gamle Sirak , der veed saa god Besked om Huusvcesenet , siger i sin Bog :  Har du Bprn , saa tugt dem , og bpi deres Hals fra Ungdommen af . " Apostelen Paulus skriver til Fcedrene :  I Fcedre , opirrer ikke Eders Bprn ; men opfpder dem i Tugt og Herrens Formaning ! " Vor Herre og Frelser har vel ikke efterladt os noget Udsagn angaaende dette <b>Kald</b> , men i dets Sted en <b>Gjerning</b> , som taler hmere end Ord . OmendskMt Guddommens Fylde ogsaa boede legemligen i Ham , medens Han var Barn , bmede Han sig dog yomygt under sine Forceldres Villie . Efter at Han i Jerusalems Tempel havde udtalt sig om sin himmelske Herkomst og svaret sin <b>Moder</b> :  Bpr jeg ikke vcere i min Faders ( Forretninger ) ? " gik Han ned med dem og kom til Nazareth og var dem underdanig . Her see I , Forceldre ; det maa saa vcere ; det er Guds Villie . Men dette see I ogsaa ved at kaste et Blik md i Barnets <b>Natur</b> . Vel er det Guds Barn , men Guds uopdragne Barn . Endnu er det ikke ved nogen klar Tro bundet til Ham ; endnu er dets Hjerte ikke klart oplyft af den Hellig-Aand ; endnu er dets Villie ikke helliget ved Guds Villie . Af Livets Begyndelse , af Udviklingens fprste Aar afhcenger hele Fremtiden . Denne kan man ikke lcegge i

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092403020'target=_>Ahlfeld, Friedrich, 1865, Menneskelivet i Guds Ords Lys</a><h4>248</h4>— i de forste Barneciar ; thi Egenvillien , Egensindigheten er allerede tidligt tilstede . Den er en Deel af Arvesynden . Den er det forste Vidnesbyrd om Hovmodet i Menneskets Indre . Daarstcib er bunden til den Unges Hjerte , siger Salomo . Vetragt dine Borns Liv . Neppe har deres Kraft udviklet sig lidt , foreud de allerede kun altfor ofte ville Andet end Fader og <b>Moder</b> ville . Endnu foreud de kunne udtrykke sin Egenvillie med Ord , legge de den for Dagen ved Fagter , Skrig og Ustikkelighed . Den findes hos Alle . Den voxer Dag for Dag . Vi kjende <b>Modre</b> , som allerede blive tyrcmniserede nf sine smnne Born . De have ikke <b>Mod</b> og Kraft til strax i de forste Aar at kjempe imod dem . Hvorledes stal det da blive i de modnere Aar , nåar Ulydigheten er bleven stor og sterk , saaledes som hos Elis Sonner? — Angrib den derfor tidligt . Medens Treet endnu er et fvagt Stud , kan det boes lige ; uaar det er blevet stort og sterkt , er der enten slet Intet mere at gjore derved , eller man knekker det . En Bek kan man afdcemme og lede ind i en nnden Bek . Forsoger mnn det Samme med en Elv , arbeider man enten forgjeves eller udsetter sig for at overskylles og bortrives af deus Vande . Den hellige Skrift , der efter Guds naadige Naadslutning stal meddele Forholdsregler for ethvert Omraade af det menneskelige Liv , legger os i vor Text ret alvorligt paa Hjertet at tåge tidligt fat paa dette <b>Arbeide</b> . Den gamle Sirat , der veed faa god Besten om Huusvesenet , siger i sin Bog : „ Hnr du Born , saa tugt dem , og boi deres Hals fra Ungdommen af . " Apostelen Paulus skriver til Fedrene : „ I Fedre , opirrer ikke Eders Born ; men opfoder dem i Tugt og Herrens Formaning ! " Vor Herre og Frelser har vel ikke efterlndt os noget Udsagn cmgacende dette <b>Kald</b> , men istedet derfor en <b>Gjerning</b> , forn tnler hoiere end Ord . Omendstjont Guddommens Fylde ogfaa boede legemligen i Hnm , medens Hnn var Barn , boede Han sig dog ydmygt under sive Foreldres Villie . Efter nt Hnn i Templet i Jerusalem havde udtnlt sig om sin himmelske Herkomst og svaret sin <b>Moder</b> : „ Bor jeg ikke vere i min Faders ( Forretninger ) ? " gik Han ned med dem og kom til Nazareth og var dem underdanig . — Der see I , I Foreldre ; det maa saa vere ; det er Guds Villie . Men dette see I ogsaa ved at kaste et Blik md i Barnets <b>Natur</b> . Vel er det Guds Baru , men Guds uopdrague Varn . Euduu er det ikke ved nogen klar Tro bundet til Ham ; endnn er dets Hjerte ikke klart oplyst af den Hellig-Aand ; endnu er dets Villie ikke helliget ved Guds Villie . Af Livets Begynnelse , af Udviklingens forste Aar afhenger hele Fremtiden . Denue kan man ikke legge i

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009072703031'target=_>Hofacker, Ludwig, 1863, Prædikener paa alle Søn- og Helligdage</a><h4>2203</h4>Men Hovedbeviset for min Paastand ligger deri : Enhver uomvendt , modvillig Synder gjsr Gud til sin Synds Ophavsmand; thi Enhver har en Undstoldning for sin Synd , og om han end skulde vcere sunket saa dybt , at han slet ikke lcengere undskyldte sin Synd for Menneskene , at han ansaae sig ligeson ; berettiget til at synde , saa undskylder han sig dog i sin <b>Samvittighed</b> for sig selv , og idet han , for at gjore sig selv retfcerdig , vcelter Skylden fra sig , saa skyder han den tilsidst over paa Gud og vil anklage Lysenes Fader som Den , der forleder ham til Synd , saa at han ikke stal kunne handle og vandre ganske saaledes, som hans <b>Samvittighed</b> fordrer af ham . Se , dette begyndte Mennesket med i Paradiset . Da Adam var falden , blev han irettesat af Gud , som sagde til ham :  Haver Du cedet af det Trce , om hvilket jeg bod Dig , at Du stulde ikke cede deraf ? " Da sagde Adam :  la , Herre , <b>Kvinden</b> , som Du gav at vcere med mig , hun , hun gav mig af det Trce , og jeg aad . " Hvorfor sagde vel Adam : „ <b>Kvinden</b> , som Du gav at viere med mig ? " Det er tydeligt , at han vilde vcelte noget af sin Skyld tilbage paa Gud ; det vil jo sige saameget som : Havde Du ikke givet mig hende , saa var jeg ikke faldet . Og saaledes forholder det fig endnu . Den Ene siger : Min Stand , mit <b>Kald</b> , mit Embede , mine Forbindelser med Menneskene , mine Omgivelser tillade ikke , at jeg omvender mig ; en Anden siger : Jeg er heftig , saaledes er mit Gemyt , min <b>Natur</b> , jeg kan derfor ikke finde mig i Christi Sagtmodighed ; en Vellystig siger : Saadan er nu engang min <b>Natur</b> . Veed Du ogsaa , hvad Du gjsr med disse Undstyldninger? Gud gjsr Du til en Synder , til en Forforer ; thi , sig mig : Hvo har sat Dig i denne Stilling , givet Dig dette Gemyt ? Det har Gud . Hvorfor ? For at Du stal synde ? Nei ! men for at det stal vcere Dig en Doelse og en Skole for Evigheden . O , en Gysen stulde fare gjennem Enhver , naar en saadan Undstyldning opstiger i Hjertet . Dette er jo ganske Satans Skik : Gud stal vcere Synderen , og Mennesket vil vcere reent — blot paa det at han ikke stal behsve at tilstaae sin Skyld . Andre igjen have andre Grunde at berolige sig med. De sige omtrent som saa : Gud er barmhjertig og vil ikke tåge det saa nsie ; han tcenker paa , at vi ere Stov : vi ere jo Allesammen Syndere , og blive aldrig fuldkomne . Hvoraf vide de da , at Gud ikke tåger det saa nsie ? Er det ikke en Mening , som kommer af deres Kjcerlighed til Synden — en elendig Slutning , som har sit Udspring fra deres fordervede Hjerte ? Vidner ikke hele Guds Ord mot denne Tale ? Der hedder det :  Ingen for »

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2010092803030'target=_>Luther, Martin, 1855, christelig Postille</a><h4>403</h4>Djcevelen eller nogen Ulykke , thi han veed , at han ved Christum er en Herre over alle Ting . At vi arme Syndere have annammet et evigt Liv og en fuldkommen Retfcerdighed , det burde jo dog idetmindste tomde en Gnist af Fryd i vore Hjerter og give os et saadant <b>Mod</b> , at vi ikke forsagede i Anfcegtning og Forfolgelse , men trodsede baade Djcevel og Verden , og sagde : " Tåge I end mit Liv — hvad er det saa meer ? bliver kun dette Barn mit , saa tåger I kun Bcelg og Skal ! Mit Tab er intet imod denne Kjcrrne og Skat , at jeg ved Christum er bleven frelst fra mine Synder, fra Guds Vrede og den evige Ded . " Men ak ! saa Faa forundre sig over de Ting , der siges om Barnet Jesus . Dog , saasom Guds Ord altid maa bringe Frugt , ere der alligevel Enkelte , som annamme det med Fryd og Forundring , nemlig Joseph og Maria . Og her straffer Evangelisten hemmeligen ledernes Vantro ; thi mange af dem vare tilstede ( det stedte jo offentlige » i Templet ) , og dog vilde Ingen troe , men de forargede sig Alle paa dette Barn . Her låre vi da , at vi gjerne stulle hore Guds Ord , thi det bliver aldrig ganske uden Frugt . Og Simeon velsignede dem . Denne Velsignelse bestod i et hjerteligt Onste om Lykke og Salighed , 3 Ere og alt Godt , og den gjceldte ikke alene Barnet , men dem alle tilhobe , Barn , Fader og <b>Moder</b> . En saadan Velsignelse kunde synes at viere en ringe Ting , thi det hore vi jo saa tit , at Folk enste hverandre Lykke og Velsignelse mdbyrdes . Men at velsigne Christum og Hans Forceldre , er ret en stor og forunderlig <b>Gjerning</b> , og det af den Grund , at Christus og <b>Natur</b> ere hinanden aldeles modsatte : Christus fordommer nemlig Alt , hvad Verden vcelger , Han giver Kors og allehaande Trcengsel , borttager al Vellyst , Rigdom og 3 Ere , og lcerer , at Alt , hvad Verdens Mennesker omgaaes med , er Daarlighed og Synd . See , det kan og vil nu Ingen lide af Ham . Derfor bander , spotter og forfolger man Christum og Alle , som hore Ham til ; saare faa Simeoner er der , som velsigne Ham , men Verden er fuld af dem , som , idetmindste i Hjertet , enste Ham al Skam og Ulykke ; thi hvo som ikke har det Sind , at han villigen giver Slip paa alle Ting og er redebon til at lide allehaande , han velsigner og lover ikke Christum lcenge , men forarger sig snart paa Ham . Derfor giver Simeon , som en Korsets Prcedikant og en Verdens Fiende , i hiin sin Velsignelse et stort og heit Erempel paa at love og cere Christum i den foragtede og forbandede

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2016042848154'target=_>Ahlfeld, Friedrich, 1866, Prædikener over Kirkeaarets Evangelier</a><h4>2097</h4>Dommen . Strommene bruse vistnok mægtigst , hvor de styrte sig i Havet , hvor de have at kjcempe med Brcendingen . Men de brnse saa allerede , medens de endnu lobe igjennem Landet , og det hoiere vaa et Sted end paa det andet . Lever du uden Herren , saa farer Domssvteroet ogfaa daglig frem mud dit Hjerte . Du foler det ofte ikke , fordi du efterhaaudeu har anskaffet dig et Pantser derimod . For den , fom itke vil vide af nogen Bod , er al Troengsel Dom . Den er et Guds Orehug mod det gamle , raadne Trce . For ham er enhver Samvittighedens Stemme en Dom . Den er en Forklang af hans Stemme , fom er Menneskets fulde , hele <b>Samvittighed</b> . For ham ere felv Guds Naadesbevisninger Dom . Han giver ham dem , for at han ikke stal kunne undskylde sig med , at Gud ikke har anvendt nok Kjærlighed til hans OvvaMelse . Dom er Foraaret , thi udenfor gronnes det til nyt Liv , i ham forbliver det dodt . Dom er Hosten , thi udenfor doer det , og i ham er det allerede uddoet . Morkt er det i Hjertet , naar Verdensglædens Lygtemænd ere udslukkede . Men denne Nat slntter sig til Gravnatten , til den evige Dods Nat . — Men hvo , fom tr o er , dommes ikke . Naade og Liv er Alt , hvad der <b>moder</b> ham . Naade og Liv er al Gloede . Du stal elske diu Frelser inderligere til Tak derfor . Naade er for ham Samvittighedsangsten . Du lærer med mere Alvor at afsijære det naturlige Menneskes vilde Grene , som endnu drage megen Saft fra det rette Liv . — Naade er for ham Korset . Det er jo Guds Kniv , som afstjærer det endnu tilbageværende Vildkjod. — Naade er Doden . Synden faaer sin Sold , for at Forbarmelfen herefter kan have fri Hoender . Hjertet er klart , og dets Lys lober nd i det salige Lys , som straaler om Guds Throne og lyser for de Benaadede . Hvo , som troer vaa Gnds Son , dommes ikke . Det er Guds Historie med hans forloste Born . — Hvis de Troendes og Vantroendes Hjerter laa « åbenlyst , da vilde det se ud som forhen i Mgyvten : her Landet Gosen , der er det lyst og klart ; hist Wgyvten , der er tykt Morke , saa at man maa famle sig frem efter Væggen . Her Alt Naade , og der Alt Dom . Livet bevidner det , nemlig det nye Liv af Gud . Bevidner det Guds <b>Gjerning</b> med dig , saa maa det ogsaa bevidne din <b>Gjerning</b> mod ham . Al Synd er Dod og bringer Dod . Lever Christus i dig , har han antcrndt nyt Liv i dig , saa maa du ogsaa leve i ham , saa maa du ogsaa vandre i et nyt Levnet . Gud kan ikke Andet , han maa elske Sine . Blomsten maa blomstre , det er dens <b>Natur</b> . Ilden maa varme , det er dens <b>Natur</b> . Er du fodt af Gud , faa kan du heller ikke Andet , du maa elske ham , du maa leve for ham , det er din nye <b>Natur</b> . En Ende maa det have med Merkets Gjerninger . Hvad den Christne gjor , kan Enhver fe . Hans

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2009092403008'target=_>Besser, W.F., 1866, St. Petri Breve</a><h4>1636</h4>den anden Side igjen befester Taalmodigheden Gudsfrygten. . . Frygter Gud ! " hertil formaner det forste Brev de Christne i Almindelighed og Undersaatterne , Tjenerne , <b>Kvinderne</b> i Besynderlighed ( 1 Br . 1 , 17 ; 2 , 17. 18 ; 3 , 2. ) . I al vor Vandel , stedse i vort jordiske <b>Kald</b> at ere den hellige Guds Villie og Bud ,  af <b>Samvittighed</b> for Gud " ( 1 Br . 2 , 19. ) at gjore vel og — om saa stal vere — at taale Uret , det er den Gndfrygtighed eller Fromhed , som vi stulle bevise i al Taalmodighed ( sml . 1 Tim . 4 , 8. ) . Ligesom i Treets Bast alle de spede Kar , som nere Treet , gaae opad , saaledes gjennemtrenge i Gndfrygtighed alle det nye Livs Krefter ( fml . V . 3. ) Christenmenneflet , saa at det Guds Menneste vorder fuldkomment ( 2 Tim . 3 , 17. ) , deelagtigt i guddommelig <b>Natur</b> . Men i Gudfrygtigheden Broderkjerlighed . Her modnes nu Treets Frugt , hvortil Alt paa det streber og virker . Det forste Brev har rigelig lert os , at Peder , hvem man rettelig har kaldt  det christelige Haabs Mand , " ikke agter Broderkjerligheden ringere end Johannes , . . den christelige Kjerligheds Mand " ( sml . iser 1 Br . 1 , 22. og 4 , 8. ) . I Broderkjerligheden beviser sig ioinefaldende og med famlet Kraft Gudfrygtigheden , den gudelige Vandel ( V . 3. ) ; vort Samfnnd med den guddommelige <b>Natur</b> aabenbares i Kjerligheden , thi  Gud er Kjerlighed , og hvo , forn bliver i Kjerlighed , bliver i Gud og Gud i ham " ( 1 Joh . 4 , 16. ) . At elste , er at hengive sig til Andre . . . Verer ' derfor Guds Efterfolgere, som elskelige Born , og omgaaes i Kjerlighed , ligesom og Christus elstede os og gav sig selv hen for os til en Gave og Slagtoffer , Gud til en velbehagelig

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2011062208060'target=_>Tholuck, August, 1847, Kristelige Andagtstimer</a><h4>2743</h4>som enHjcelp , fordi der er givet <b>Qvinden</b> , hvad den mandlige <b>Natur</b> fattes eller hvad den ikke let kan forrette . <b>Qvinden</b> er , som den hellige Skrift af mere end een Grund har kaldt dette Kjon , ~ det svage Vcerktoj " ( l Ped . 3 , 7 ) . Thi hverken er hendes Legeme skabt til saa stort <b>Mod</b> eller saa besvcerligt Arbejde , wm det mandlige Legeme ; heller ikke er hendes Aand af Gud bestemt til at udforsse Viisdommens Dybder eller at fore Regimentets Tsmme . Men hvad kristelige <b>Qvinder</b> fremfor alt stulle have til Arvegods , og hvad der allerede indtil en vis Grad er givet dem i deres naturlige Anlcrg , det er ~ Hjertcts stjulte Menneske uforstyrret med blid og stille Aand " ( 1 Ped . 3 , 4 ) , for at de i <b>ydmyg</b> Kjcrrlighed og med stor Qmhed stulle stalte og valte bjemmc og bestvre Huset , medens Manden paa Gadcr og Strceder arbejdcr for Huset eller ogsaa tager sig af Landet , som allcrede Salomon striver : ~ Hvem en dydig Qvindc er bleven til Deel , ham er hun langt mere vcrrd end Perler , hendcs Mands Hjerte tsr forladc sig paa hende , og Vinding vil han ikke fattes , hun forvolder ham Glcede og ingen Sorg , saalcenge hun lever , bun omgaaes med Ult » og Hsr og arbejder gjerne med sine Hunder — men hendes Mand er berømt i Portene , naar han siddcr dlandt Landets Wldste " ( Ordspr . 3 ! , 10 - 13. 23 ) . Saaledes ssrivcr ogsaa Apostelen : ~ En Qviude bor hore efter i Stilhed med al Underdanighed , jeg tillader ikke en <b>Qvinde</b> at undervise eller vcere Mandens Herre , men at vcere stille " , og : , Hun stal blive salig ved at fode Born , naar hun bliver i Tro og i Hclliggjsrelse samt i Tugt " ( 1 Tim . 2 , 11 - 12. 15 ) . Ved al stig Bistand hjemme bliver en Wgtchustru i Ordets sande Betydning Mandens anden Halvdeel , og medens han , som hans Stand og <b>Kald</b> forer det med sig , maae fcrrdcs den storste Deel af Dagen ude af Huset hos Folk paa Gadcn , kan han nu voere rolig og ved godt <b>Mod</b> , da han veed , at hans anden Halvdee ! bliver tilbage i Hjemmet , sor at varetage Husets Tarv . Ingen Ven vilde , selv om han mecnte det nok saa godt , i denne seendc kunne gjore Qvindens Hjcrlp overstsdig eller trcrde i Stedet for en anden Halvdcel . — Ligesom hun nu saaledes ved sin stille Vandel og sit tugtige Regimente i det udvortes Mennestes Anliggcnder bliver hans Hjcrlp , saaledes er hun ogsaa ved sin <b>qvindelige</b> <b>Natur</b> mere end nogensindc nogen Ven cgnet til Mantilforordnet

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013100708190'target=_>Funcke, Otto, 1877, Apostelen Paulus til Søs og til Lands</a><h4>156</h4>eller at vederkvoege sig i storartet Naturskønhed , - alt sligt er ncegtet de fleste her paa Jorden . Derfor har da Gud , alle Uanders Fader , der kjender det menneskelige Hjcerte til Bunds og har Hjcerte for Vaskerkonerne saa vel som for Fyrstinderne , ftrget for , at ingen af os alle stal mangle den nodvendige Forandring. Hvorledes da ? Han har givet os Sabbaten , S ond agen , paa hvilken vi stal Pile fra al vor <b>Gjerning</b> og gloede os over al hans <b>Gjerning</b> , gloede os ved Samfundet med vore Medmennesker , hans Bprn , og saaledes een Gang hver Uge ligesom holde Helligaften , ftr det tilkommende , falige Fredsliv oprinder . Ak , gid dog vor urolige og rastløse Tids Mennesker , der jager fra det ene til det andet , vilde erkjende den Velgjerning , der bevifes os i Sabbaten ! Sandelig , hvor der ingen Søndag holdes , der bliver Livet en endeløs „Jammerdal", fom mange Mennester kalder det , men fom det ikke er Guds Villie , at det skulde vcere . Ve det arme Menneste , der henslceber sin Tid som et Arbejdsdyr i fortærende og uendelig Ensformighed i hundrede og atter hundrede Dage , den ene som den anden . Menneskenes Daarstab , der altid stal kritisere Guds Visdom , har ment , at det dog maatte voere gavnligere for Legeme og Sjoel og mere overensstemmende med deres <b>Natur</b> , at man levede den ene Dag ganske som den anden , og at man altsaa i Stedet for at hvile hver syvende Dag hvilede en syvende Del af hver Dag og hver Dag gjorde fig en Fornøjelse og spgte lidt Selskabelighed . Den , der prpver det , vil snart opdage , hvor drcebende kjedeligt et saadant Liv er ! Nej , netop den fuldstændige Forandring , som Spndagen bringer , da Du for een Gangs Skyld med en god <b>Samvittighed</b> tpr rive Dig ud af den Tummel , der omgiver Dig , da Du med <b>Hustru</b> og Bprn tM glcede Dig over din Tilværelse ligesom i en Oase i ørkenen under Figentræer og Vinranker , da Du faar T : d til rigtig at tcenke paa , at Du er bestemt til at vcere et Guds Barn og hans Arving , og svinger din Sjoel , der er bleven nceret med Guds Ord og BM , op trl Faderen

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2014010908003'target=_>Friis, J.A., 1880, Sommer i Finmarken, Russisk Lapland og Nordkarelen</a><h4>754</h4>i Almindelighed Vaarfisket sluttet , og den fiskende Finnebefolkning vender nu tilbage til Vadsø for at faa Klarering af sine Redere eller Medfiskere . Har Fisket været rigt , hersker der da en Uges Tid i Eegelen et glad Liv i Kvænebyen . Matti og Jussi og Pekka og Piera fra Haapajårvi , Kalajoki eller Kuusamo og andre Sogne i det nordlige Finland eller de starovjerske Kareler Jeremei , Ivan , Seder o . fl. fra Pååjårvis og Kuitojårvis Bredder , der kom næsten nøgne til Vadsø , forsultne og uden en Skilling i Lommen , have nu gode Klæder paa en velnæret Krop og blanke Sølvdalere i Lommen . Men disse ere ogsaa ofte surt fortjente . <b>Arbeidet</b> paa Havet er haardt , Blæsten bidende og Bekvemmelighederne i Land uhyre tarvelige . Intet Under derfor , om Jussi og Matti efter Tilbagekomsten til den hyggelige Kvænby føle sig fristede til at opfriske Livsaanderne med en Slurk af det midt i al Fiskeluft lifligt duftende Brændevin og Rom . Dog kun en liden Del af Gevinsten maa anvendes , kun en ganske liden Übetydelighed , saavidt at de noget stivnedé og støre Lemmer faa sin Smidighed igjen . Besten af de blanke Penge maa spares og medbringes hjem , hvor kanske en <b>Hustru</b> med en Flok Unger , klædte i Traser og forsultne , længselsfuldt vente paa sin hjemvendende Fader . Eller kanske der før Afreisen blev indgaaet en Trolovelse og sluttet en Kontrakt om et lidet « Torp » , som skal betales ved Hjemkomsten , eller kanske en gammel <b>Moder</b> venter sin Søn pengerig hjem igjen for at redde hende fra Hungersnøden . Imidlertid , eller hvordan det end er , en halv Pægel kan dog den bedste Fader eller Søn tåge sig med god <b>Samvittighed</b> . Til deres Bos maa man ogsaa sige , at de Fleste lade det blive dermed og begive sig strax paa Hjemveien med største Delen af sin Fortjeneste . Men Undtagelser findes ogsaa , og desværre, den ædle Drik er deroppe , ikke mindre end andre Steder , af den <b>Natur</b> , at jo mere man drikker deraf , des mere tørster man , og Norges Love ere i saa Maade maaske liberalere end Finlands , at man ustraffet kan tåge sig en Tåar over Tørsten . Hvorfor skulde man da ikke idetmindste gjøre et Forsøg paa at slukke den ? Man tåger derfor endnu en halv Bægel , derpaa en hel og saa begiver Jussi og Piera , glade i Hu og Arm i Arm , sig hen til Spelar- Matti eller en eller anden glad Kammerat . Der spilles Kort , og nu indsamler Spelar-Matti og hans Kammerater mange af de surt fortjente Dalere ; thi ihærdig i Alt , saa er Finnen ogsaa ihærdig i Spil og giver ikke tabt ved de første uheldige Forsøg . Men paa

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013121238005'target=_>Friis, J.A., 1871, Sommer i Finmarken, russisk Lapland og Nordkarelen</a><h4>846</h4>sluttet , og den fiskende Finnebefolkning vender nu tilbage til Vadsø for at faa Klarering af sine Redere eller Mediiskere . Har Fisket været rigt , hersker der da en Uges Tid i Regelen et glad Liv i Kvænebyen . Matti . og Jussi og Pekka og Piera fra Haapajårvi , Kalajoki eller Kuusamo og andre Sogne i det nordlige Finland eller de starovjerske Kareler Jeremei , Ivan , Seder o . fl. fra Pååjårvis og Kuitojårvis Bredder , der kom næsten nøgne til Vadsø , forsultne og uden en Skilling i Lommen , , have nu gode Klæder paa en velnæret Krop og blanke Sølvdalere i Lommen . Men disse ere ogsaa ofte surt fortjente . <b>Arbeidet</b> paa Havet er haardt , Blæsten bidende og Bekvemmelighederne i Land uhyre tarvelige . Intet Under derfor , om Jussi og Matti efter Tilbagekomsten til den hyggelige Kvænby føle sig fristede til at opfriske Livsaanderne med en Slurk af det midt i al Fiskeluft lifligt duftencle Brændevin eller Rom , Dog kun en liden Del af Gevinsten maa anvendes , kim en ganske liden Übetydelighed , saavidt at de noget stivnede og støre Lemmer faa sin Smidighed igjen . Resten af de blanke Penge maa spares og medbringes hjem , hvor kanske en <b>Hustru</b> med en Flok Unger , klædte i Traser og forsultne , lærigselsfuldt vente paa sin hjemvendende Fader . Eller kanske der før Afreisen blev sluttet Kontrakt om et lidet „ Torp " , som skal betales ved Hjemkomsten, eller kanske en gammel <b>Moder</b> venter sin Søn pengerig hjem igjen for at redde hende fra Hungersnøden . Imidlertid , eller hvordan det end er , en halvPægel kan dog den bedste Fader eller Søn tåge sig med god <b>Samvittighed</b> . Til deres Ros maa man ogsaa sige , at de Fleste lade det blive dermed og begive sig strax paaHjemveienmed største Delen af sin Fortjeneste . Men Undtagelser findes ogsaa , og desværre , den ædle Drik er deroppe , ikke mindre end andre Steder , af den <b>Natur</b> , at jo mere man drikker deraf , / des

<h4> <a href='https://urn.nb.no/URN:NBN:no-nb_digibok_2013020524029'target=_>Holberg, Ludvig, 1834, Niels Klims Reise under Jorden</a><h4>911</h4>Men tkke lange derefter blcv der en strakkelig Skrigen og Tuden i Kokkenet , saa det stingrede i hele Huset . Jeg wb ud , og saa med stsrste Bestyrtelse min Herr Cujon i fuldt <b>Arbeide</b> med at msrprygle sin <b>Kone</b> og Tjenestepigerne ; men , aa snart han saa mig , sik han Been og tog Flugten . leq vendte mig derpaa til <b>Konen</b> og Pigerne , som grad , og wurgtc dem , hvad for en Misgierning de havde siden de havde bragt denne sagtmodige Mand til saa gyselig en Vrede . De stede lange med Sinene Mede paa Jorden , og svarede mig ikke et Ord , som om de frygtede for at lade Ng marke med deres Bedrsvelse ; men langt om lamge , da ) eg baade med Bonncr og Truster blev ved at spsrge , brod <b>Konen</b> omsider ud i folgende Ord . - „ Du synes ikke , kiare . Fremmede , at have megen Kundskab til Menneskerne . Borgerne her , som ikke tor see nogen bcvabnet Fiende under « Dine , og , ved den mindste Larm udenfor , gierne krsbc i « et Musehul af Forstrakkelse , herffe alle i deres Kokkener , ° og holde tyrannist Hus med os svage Frucntimre . <b>Mod</b> « bevæbnede Folk fore de aldrig Krig ; det er kun Übevæbnede . de binde an med. Derfor er vort Land ogsaa bestandigt » ud , at for Naboernes Spot og Udplyndring . I det ncrste « Rige herved , som vi maac betale Skat til , ere Mandene - ganske anderledes beskafne ; de slaaes aldrig uden med bevab«nede Ficnder , herske udenfor Huset , og lystre hjemme . " Jeg beundrede denne Kones Forstand , og fandt , at hun var en bedre Skabne rard . Og da jeg siden i Tidens Lcrngde havde gjort mig noiere bekiendt med den menneskelige <b>Natur</b> , maatte jeg tilstaae m , ' g selv , at hun havde havt




In [120]:
with open('tema_lordag_12_14-20.html', 'w', encoding='utf-8') as f:
    f.write(res)